In [1]:
# 1. Enable Internet in the Kernel (Settings side pane)

# 2. Curl cache may need purged if v0.1.6 cannot be found (uncomment if needed). 
# !curl -X PURGE https://pypi.org/simple/kaggle-environments

# ConnectX environment was defined in v0.1.6
!pip install 'kaggle-environments>=0.1.6'

from kaggle_environments import evaluate, make, utils
from numpy import reshape,argmax

env = make("connectx", debug=True)
env.render()

     |████████████████████████████████| 1.2 MB 4.1 MB/s 
     |████████████████████████████████| 56 kB 4.3 MB/s 
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 2.6.0
    Uninstalling jsonschema-2.6.0:
      Successfully uninstalled jsonschema-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbclient 0.5.4 requires jupyter-client>=6.1.5, but you have jupyter-client 5.3.5 which is incompatible.
Loading environment football failed: No module named 'gfootball'


In [2]:
#@title ***One-step Look Ahead*** - test_agent_1
def test_agent_1(obs, config):
    import random
    import numpy as np
    
    #---------------------------------------------------------------
        # Calculates score if agent drops piece in selected column
    def score_move(grid, col, mark, config):
        next_grid = drop_piece(grid, col, mark, config)
        score = get_heuristic(next_grid, mark, config)
        return score

    # Helper function for score_move: gets board at next step if agent drops piece in selected column
    def drop_piece(grid, col, mark, config):
        next_grid = grid.copy()
        for row in range(config.rows-1, -1, -1):
            if next_grid[row][col] == 0:
                break
        next_grid[row][col] = mark
        return next_grid

    # Helper function for get_heuristic: checks if window satisfies heuristic conditions
    def check_window(window, num_discs, piece, config):
        return (window.count(piece) == num_discs and window.count(0) == config.inarow-num_discs)

    # Helper function for get_heuristic: counts number of windows satisfying specified heuristic conditions
    def count_windows(grid, num_discs, piece, config):
        num_windows = 0
        # horizontal
        for row in range(config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[row, col:col+config.inarow])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # vertical
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns):
                window = list(grid[row:row+config.inarow, col])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # positive diagonal
        for row in range(config.rows-(config.inarow-1)):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row+config.inarow), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        # negative diagonal
        for row in range(config.inarow-1, config.rows):
            for col in range(config.columns-(config.inarow-1)):
                window = list(grid[range(row, row-config.inarow, -1), range(col, col+config.inarow)])
                if check_window(window, num_discs, piece, config):
                    num_windows += 1
        return num_windows
    
        # Helper function for score_move: calculates value of heuristic for grid
    def get_heuristic(grid, mark, config):
        A = 10000
        B = 1000
        C = 50
        D = -500
        E = -100000
        num_twos = count_windows(grid, 2, mark, config)
        num_threes = count_windows(grid, 3, mark, config)
        num_fours = count_windows(grid, 4, mark, config)
        num_twos_opp = count_windows(grid, 2, mark%2+1, config)
        num_threes_opp = count_windows(grid, 3, mark%2+1, config)
        score = A*num_fours + B*num_threes + C*num_twos + D*num_twos_opp + E*num_threes_opp
        return score
    #-----------------------------------------------
     
    valid_moves = [c for c in range(config.columns) if obs.board[c] == 0]
    # Convert the board to a 2D grid
    grid = np.asarray(obs.board).reshape(config.rows, config.columns)
    # Use the heuristic to assign a score to each possible board in the next turn
    scores = dict(zip(valid_moves, [score_move(grid, col, obs.mark, config) for col in valid_moves]))
    # Get a list of columns (moves) that maximize the heuristic
    max_cols = [key for key in scores.keys() if scores[key] == max(scores.values())]
    # Select at random from the maximizing columns
    return random.choice(max_cols)

In [9]:
#@title ***PoZero Approach*** - test_agent_2
from kaggle_environments.envs.connectx.connectx import play,is_win
import time
import random
import torch
import numpy as np
from numba import jit
class Alpha_Net(torch.nn.Module):
    def __init__(self):
        super(Alpha_Net,self).__init__()
        self.block1 = torch.nn.Sequential(
        torch.nn.Conv2d(2,20,kernel_size=2,padding=0,stride=1),
        torch.nn.BatchNorm2d(20),
        torch.nn.LeakyReLU(),
        torch.nn.Conv2d(20,40,kernel_size=3,padding=0,stride=1),
        torch.nn.BatchNorm2d(40),
        torch.nn.LeakyReLU(),
        torch.nn.Conv2d(40,120,kernel_size=3,padding=0,stride=1),
        torch.nn.BatchNorm2d(120),
        torch.nn.LeakyReLU(),
        )
        self.block2 = torch.nn.Sequential(
        torch.nn.Linear(240, 128),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(128, 64),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(64, 7),
        )
        self.t_out = torch.nn.LogSoftmax(dim=1)
        self.e_out = torch.nn.Softmax(dim=1)
        self.block3 = torch.nn.Sequential(
        torch.nn.Linear(240, 64),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(64, 32),
        torch.nn.LeakyReLU(),
        torch.nn.Linear(32, 1),
        torch.nn.Tanh()
        )

    def forward(self, x):
        x = self.block1(x)
        x = x.view(-1,240)
        x1 = self.block2(x)
        if self.training:
            x1 = self.t_out(x1)
        else:
            x1 = self.e_out(x1)
        x2 = self.block3(x)
        x = torch.cat([x1,x2],dim=1)
        return x
@jit
def opt_tp(policy,Q,ramda):
    alpha_min = np.max(Q+ramda*policy)
    alpha_max = np.max(Q)+ramda
    tp0 = np.sum(ramda*policy/(alpha_min-Q))-1
    tp1 = np.sum(ramda*policy/(alpha_max-Q))-1
    if tp0 >= 0 and tp1 >= 0:
        alpha0 = alpha_max
    elif tp0 < 0 and tp1 < 0:
        alpha0 = alpha_min
    else:
        for i in range(100):
            alpha0 = (alpha_max + alpha_min) / 2
            tpr = np.sum(ramda*policy/(alpha0-Q))-1
            if np.abs(tpr) < 1e-4:
                break
            if tpr >= 0:
                alpha_min = alpha0
            else:
                alpha_max = alpha0
    target_policy = ramda*policy/(alpha0-Q) / np.sum(ramda*policy/(alpha0-Q))
    return target_policy
def playout(board0,config,num=21):
    score = 0.0
    turn0 = sum([1 if p != 0 else 0 for p in board0])%2 + 1
    for n in range(num):
        board = board0[:]
        for t in range(42):
            turn = sum([1 if p != 0 else 0 for p in board])%2 + 1
            action = random.choice([c for c in range(7) if board[c] == 0])
            play(board,action,turn,config)
            if sum([1 if p != 0 else 0 for p in board]) >= 42:
                break
            if is_win(board,action,turn,config,True):
                if turn0 == turn:
                    score += 1
                else:
                    score -= 1
                break
    return score / num
def alpha_MCTS(net,board,config,start_time=None,info=None,expand=False,root=False,C=1.25,expand_threshold=1,time_lim=2.0,gamma=0.98,steps=1000000):
    if info == None:
        b1 = np.array([[[1 if p == 1 else 0 for p in board]]]).reshape(1,1,6,7)
        b2 = np.array([[[1 if p == 2 else 0 for p in board]]]).reshape(1,1,6,7)
        board2 = np.concatenate([b1,b2],axis=1)
        board2 = torch.from_numpy(board2).float()
        pred = net(board2).detach().numpy()[0]
        policy = pred[:7]
        value = pred[7]
        count = [0,0,0,0,0,0,0]
        info_each = [0,0,0,0,0,0,0]
        Q = np.zeros(7)
        selectable = np.ones_like(Q)
        for pin in range(7):
            if board[pin] != 0:
                Q[pin] = -1
                selectable[pin] = 0
    else:
        policy = info['policy']
        info_each = info['info_each']
        count = info['count']
        Q = info['Q']
        value = info['value']
        selectable = info['selectable']
        playout_value = info['playout_value']
    turn_count = sum([1 if p != 0 else 0 for p in board]) + 1
    turn = sum([1 if p != 0 else 0 for p in board])%2 + 1
    if root:
        start_time = time.time()
        for t in range(steps):
            if sum(count) < 1:
                target_policy = policy
            else:
                ramda = C*np.sqrt(sum(count))/(7+sum(count))
                target_policy = opt_tp(policy,Q,ramda)
            child_index = random.choices([0,1,2,3,4,5,6],weights=list(target_policy),k=1)[0]
            if board[child_index] != 0:
                Q[child_index] = -1
            else:
                next_board = board[:]
                if is_win(next_board,child_index,turn,config,False):
                    Q[child_index] = 1
                else:
                    play(next_board,child_index,turn,config)
                    fillness = sum([1 if p != 0 else 0 for p in next_board])
                    if fillness == 42:
                        Q[child_index] = 0
                    else:
                        if count[child_index] >= expand_threshold:
                            scc, ie = alpha_MCTS(net,next_board,config,start_time,expand=True,info=info_each[child_index])
                            Q[child_index] = -scc
                            info_each[child_index] = ie
                        else:
                            if count[child_index] == 0:
                                scc, pol, Q_e, selectable_e, pee = alpha_MCTS(net,next_board,config,start_time,info=None)
                                vava = scc*(1-turn_count/42) + pee*(turn_count/42)
                                Q[child_index] = -vava
                                info_each[child_index] = {'policy':pol,'info_each':[0,0,0,0,0,0,0],'count':[0,0,0,0,0,0,0],'Q':Q_e,'value':scc,'selectable':selectable_e,'playout_value':pee}
            count[child_index] += 1
            if time.time()-start_time >= time_lim:
                break
        ramda = C*np.sqrt(sum(count))/(7+sum(count))
        target_policy = opt_tp(policy,Q,ramda)
        return target_policy
    else:
        if expand:
            if sum(count) < 1:
                target_policy = policy
            else:
                ramda = C*np.sqrt(sum(count))/(7+sum(count))
                target_policy = opt_tp(policy,Q,ramda)
            child_index = random.choices([0,1,2,3,4,5,6],weights=list(target_policy),k=1)[0]
            if board[child_index] != 0:
                Q[child_index] = -1
            else:
                next_board = board[:]
                if is_win(next_board,child_index,turn,config,False):
                    Q[child_index] = 1
                else:
                    play(next_board,child_index,turn,config)
                    fillness = sum([1 if p != 0 else 0 for p in next_board])
                    if fillness == 42:
                        Q[child_index] = 0
                    else:
                        if count[child_index] >= expand_threshold:
                            scc, ie = alpha_MCTS(net,next_board,config,start_time,expand=True,info=info_each[child_index])
                            Q[child_index] = -gamma*scc
                            info_each[child_index] = ie
                        else:
                            if count[child_index] == 0:
                                scc, pol, Q_e, selectable_e, pee = alpha_MCTS(net,next_board,config,start_time,info=None)
                                vava = scc*(1-turn_count/42) + pee*(turn_count/42)
                                Q[child_index] = -gamma*vava
                                info_each[child_index] = {'policy':pol,'info_each':[0,0,0,0,0,0,0],'count':[0,0,0,0,0,0,0],'Q':Q_e,'value':scc,'selectable':selectable_e,'playout_value':pee}
            count[child_index] += 1
            if sum(count) < 1:
                return value, {'policy':policy,'info_each':info_each,'count':count,'Q':Q,'value':value,'selectable':selectable,'playout_value':playout_value}
            ramda = C*np.sqrt(sum(count))/(7+sum(count))
            target_policy = opt_tp(policy,Q,ramda)
            re_Q = np.array(Q) * selectable * target_policy / np.sum(selectable * target_policy)
            r = np.power(2,sum(count)-1)/(1+np.power(2,sum(count)-1))
            vava = value*(1-turn_count/42) + playout_value*(turn_count/42)
            return np.sum(re_Q)*r + vava*(1-r), {'policy':policy,'info_each':info_each,'count':count,'Q':Q,'value':value,'selectable':selectable,'playout_value':playout_value}
        else:
            playout_value = playout(board,config)
            return value, policy, Q, selectable, playout_value
net = Alpha_Net()
net.state_dict()['block1.0.weight'][:] = torch.tensor([[[[0.494994,0.510215],[0.43694896,-0.11027781]],[[-1.0437814,-0.03083113],[-0.17431441,-1.1425921]]],[[[0.2780717,0.157697],[-0.22472934,0.660709]],[[-1.0263442,-1.3063086],[0.12910013,-0.43064612]]],[[[-0.03836016,-0.18587069],[1.1735188,0.698932]],[[-0.0126221,-0.22315535],[0.5514721,-0.6166628]]],[[[0.03914747,1.0446723],[-0.36314613,0.75670743]],[[0.08550641,-0.74836814],[-0.20769116,-0.03782699]]],[[[0.23775905,-0.4916376],[-0.3171383,-0.62170565]],[[-0.28140697,-0.28243718],[0.56440884,-0.40798193]]],[[[-0.9859528,-0.35126922],[-0.06898504,0.21818103]],[[-0.14352338,-0.05584137],[1.1489925,0.20174709]]],[[[-0.59407926,0.54936063],[-0.32634845,0.01355284]],[[-0.5074589,1.0521286],[0.75657606,0.17462303]]],[[[-0.5026645,-0.05741369],[0.5557679,0.7667674]],[[-0.51057583,-0.3015836],[0.2681827,0.81217897]]],[[[0.47402698,-0.9083387],[0.56562865,-0.63138574]],[[0.4170411,-0.52459764],[-0.32844955,-0.29416034]]],[[[0.73627573,0.25919217],[-0.8525568,0.09815661]],[[0.67675847,-0.02335458],[0.44004932,-0.7899481]]],[[[-0.2892524,-0.5129089],[0.1294404,-0.46544123]],[[0.9714843,0.5376494],[0.15523912,0.7570932]]],[[[-0.24800275,0.41680053],[0.5798326,0.595769]],[[-0.81849694,0.3602438],[-0.40271565,-0.84750706]]],[[[0.38313428,0.7663956],[0.51795745,-0.57873607]],[[-0.11694335,0.03078658],[-1.0589381,-0.2637813]]],[[[-0.70078605,0.50425124],[0.5756955,-0.39414302]],[[-0.9333115,-0.36590776],[0.389459,-0.27370667]]],[[[-0.4694921,-0.53205794],[0.7058761,-0.30063078]],[[-0.5611944,0.7654916],[-0.14650589,0.51238436]]],[[[-0.15109417,0.12326165],[-1.0937922,-0.38268816]],[[0.15088023,-0.16974364],[-0.13752809,0.8969189]]],[[[-0.06399372,-0.6811932],[-0.43638882,-0.47168785]],[[0.44838035,-0.08628332],[0.7503476,0.77161473]]],[[[0.88735414,0.10167708],[0.24466506,0.8934199]],[[-0.3343072,-0.04815819],[-0.87906474,0.01608844]]],[[[-0.22575381,0.6470207],[-0.09993661,-0.6066506]],[[0.5373617,0.30434412],[-0.0978016,-1.0754868]]],[[[-0.93681407,0.13308094],[-0.5689326,-0.86154276]],[[0.7534786,0.11316171],[0.33851033,0.1760125]]]])
net.state_dict()['block1.0.bias'][:] = torch.tensor([0.010093856,0.0021289827,-0.0033054121,0.003545577,-0.005658894,0.0045790006,0.0005714812,-0.00016254885,0.0034296345,-0.005213071,-0.0010690996,-0.0028761833,0.0016035859,0.0036434087,-0.00077028316,0.005742198,0.0014562303,-0.0043495204,-0.0027937463,0.008342019])
net.state_dict()['block1.1.weight'][:] = torch.tensor([1.3495404,1.3594335,1.331093,0.9853874,1.4151697,1.1542124,0.86250186,1.0523511,1.2440994,1.1440706,1.232832,1.1618015,1.2185249,1.1799058,1.0524304,1.5582947,1.2571796,1.2817482,1.1115214,1.5299228])
net.state_dict()['block1.1.bias'][:] = torch.tensor([0.00740379,-0.03745603,-0.6011364,0.44238287,-0.5662089,-0.31787992,-0.17804962,-0.73606163,-0.1881759,0.16005284,-0.019066347,0.033038143,0.055289503,-0.181367,-0.08503593,-0.05980549,-0.4489608,0.25044438,0.058748875,0.16745408])
net.state_dict()['block1.1.running_mean'][:] = torch.tensor([-0.17381473,-0.2163657,0.29700166,0.09673221,-0.27322054,0.07568966,0.17517045,0.28070173,-0.19814232,-0.003963254,0.19226971,-0.036069848,-0.11313094,-0.13800532,0.03977813,-0.1568562,0.04827451,0.14714144,-0.19217572,-0.17503345])
net.state_dict()['block1.1.running_var'][:] = torch.tensor([0.29409117,0.43563694,0.345193,0.2771384,0.25881404,0.18769652,0.25706837,0.14119941,0.3132755,0.320904,0.31519347,0.29437202,0.31214198,0.14642547,0.24613537,0.3146951,0.25581297,0.27058733,0.14201693,0.32870212])
net.state_dict()['block1.1.num_batches_tracked'] = torch.tensor(305600)
net.state_dict()['block1.3.weight'][:] = torch.tensor([[[[-2.86340833e+00,-3.77547354e-01,-7.71600008e-01],[-1.86495960e-01,-2.22272754e-01,-1.57905042e-01],[-1.24550559e-01,-7.59165943e-01,-1.93555385e-01]],[[-2.07438850e+00,-5.24338841e-01,8.46488923e-02],[1.36322960e-01,3.30738008e-01,-1.50100127e-01],[-8.37224901e-01,6.55002117e-01,-4.44077641e-01]],[[1.80550265e+00,2.81701952e-01,3.57503921e-01],[2.94603663e-03,-1.07579917e-01,1.89026326e-01],[5.81528068e-01,-8.06482807e-02,4.35527265e-01]],[[4.27324504e-01,1.17648020e-01,-1.05341278e-01],[2.57124901e-01,-3.14711511e-01,7.40907330e-04],[-4.17083889e-01,1.59745887e-01,1.59245208e-02]],[[1.33234099e-01,-2.02458516e-01,1.91670924e-01],[1.13055505e-01,2.83389866e-01,5.45155346e-01],[-3.15717191e-01,6.58975244e-01,-1.67261809e-01]],[[-1.81400999e-01,-6.68029666e-01,3.47464353e-01],[-1.13514714e-01,3.29955444e-02,2.30526447e-01],[-3.78949404e-01,2.15315625e-01,-1.04370587e-01]],[[-7.52340734e-01,2.07638010e-01,1.46043688e-01],[9.49813128e-02,-4.98013198e-01,-7.99177960e-02],[-1.89029366e-01,-3.10315430e-01,-2.90257335e-01]],[[-9.61432815e-01,2.98225909e-01,-5.04530333e-02],[-2.21576646e-01,5.06831706e-01,-1.56623900e-01],[-5.42249918e-01,1.56430259e-01,3.38773072e-01]],[[2.20631957e-01,2.42169768e-01,5.33801675e-01],[3.13801765e-02,3.56037736e-01,5.20883910e-02],[5.39480150e-01,6.40644073e-01,-6.29152209e-02]],[[6.56568050e-01,1.59167737e-01,4.22272265e-01],[2.34253004e-01,-1.17088750e-01,-2.11034074e-01],[-2.50721604e-01,3.34469914e-01,1.73802391e-01]],[[2.46333957e+00,6.98306039e-02,6.33856893e-01],[-1.05131894e-01,1.46288380e-01,-1.19205162e-01],[-1.47653148e-01,9.86440778e-02,-3.01043451e-01]],[[-1.03054750e+00,-1.73138604e-02,-5.81124067e-01],[-3.62257361e-01,-7.07743466e-02,-1.38691530e-01],[-4.80908632e-01,1.75131604e-01,-4.66414317e-02]],[[-8.75444412e-01,5.56427658e-01,2.31712759e-01],[4.13622051e-01,-2.18576759e-01,1.85682610e-01],[5.35728395e-01,-4.91390854e-01,-8.61392796e-01]],[[-6.09619915e-01,-5.84958613e-01,-1.31858692e-01],[1.32214949e-01,-7.15912521e-01,2.31968220e-02],[3.80250871e-01,-2.64786303e-01,-6.13912702e-01]],[[-1.38062406e+00,-4.73517179e-02,1.81615710e-01],[-7.78444886e-01,-3.00969899e-01,-6.03348792e-01],[1.37657925e-01,-4.93260592e-01,8.20823759e-02]],[[-6.27154052e-01,-5.87699771e-01,-2.71620061e-02],[-2.30074048e-01,2.13497803e-02,2.34029680e-01],[6.37253940e-01,-5.29129744e-01,1.84685037e-01]],[[1.13814533e+00,1.09894127e-01,3.29248071e-01],[3.98644984e-01,1.90090343e-01,9.48645324e-02],[-3.27076048e-01,1.67707652e-01,-7.89772034e-01]],[[-1.86351985e-01,2.38231614e-01,9.30898190e-02],[4.13723826e-01,7.63446316e-02,5.49346535e-03],[-8.52054209e-02,-6.14440367e-02,-1.47437945e-01]],[[9.92204070e-01,5.84233642e-01,-5.20104349e-01],[-1.33724764e-01,6.35128558e-01,5.74751556e-01],[1.86066985e-01,3.12757879e-01,-7.13711023e-01]],[[1.38788080e+00,-7.99633324e-01,6.49073601e-01],[-5.87114990e-01,-3.09868902e-01,2.07165375e-01],[1.47371054e-01,2.04105124e-01,-3.26184392e-01]]],[[[7.60503769e-01,-2.70908233e-02,6.17357314e-01],[3.07458669e-01,2.20502660e-01,-6.08918309e-01],[1.17048132e+00,-6.20067008e-02,3.19759697e-01]],[[5.03144622e-01,7.34260539e-03,-2.75314003e-01],[1.75174013e-01,6.38026237e-01,-5.93842685e-01],[1.60910264e-01,7.63377488e-01,6.29432738e-01]],[[8.07665706e-01,-4.60976332e-01,2.60153383e-01],[-1.02358878e-01,1.57319546e-01,5.55745721e-01],[4.46675479e-01,-1.06177628e+00,1.19952202e-01]],[[-3.34211648e-01,-1.55855954e-01,-1.09729148e-01],[-2.27013737e-01,3.15442264e-01,-3.47999483e-01],[-9.04637218e-01,4.20676798e-01,-3.21249366e-01]],[[-1.77224368e-01,-4.40555483e-01,2.92590797e-01],[-9.62019682e-01,7.93180585e-01,5.71588993e-01],[-5.03893852e-01,3.08874726e-01,-9.89657223e-01]],[[-4.04947996e-02,3.09331715e-01,4.91059273e-02],[1.11811319e-02,-2.29513109e-01,-3.45633745e-01],[2.59876728e-01,3.26698750e-01,3.25987041e-01]],[[1.92822739e-01,5.06398678e-01,2.65368342e-01],[-8.47910270e-02,4.40704882e-01,-1.63977191e-01],[2.55832851e-01,-5.53501368e-01,-2.84942925e-01]],[[1.43554199e+00,5.72868347e-01,1.94516614e-01],[-1.72111794e-01,1.99748918e-01,1.98832620e-02],[3.07544470e-01,-7.15048552e-01,6.01637363e-01]],[[-4.56360042e-01,9.02382135e-02,-5.22618066e-04],[6.17244720e-01,-2.17537787e-02,-1.49017707e-01],[8.43111753e-01,-5.12793481e-01,-1.78262889e-01]],[[-1.06809378e+00,3.98393095e-01,-2.42429435e-01],[-3.95455807e-01,-1.67369962e-01,-6.06014252e-01],[-1.16514766e+00,5.44935048e-01,1.82376549e-01]],[[-1.68867385e+00,1.42052874e-01,2.63554275e-01],[-2.30497643e-01,-2.72991121e-01,-7.90532529e-01],[1.35650948e-01,-3.60029340e-01,-5.93065619e-01]],[[4.32605743e-01,-2.61398703e-01,-8.99718106e-02],[3.31762642e-01,1.29286647e-01,1.73869319e-02],[-7.76236236e-01,-4.42727149e-01,2.15324983e-02]],[[9.93267596e-02,1.23739645e-01,1.46537527e-01],[1.49835616e-01,-6.93015039e-01,-6.78403854e-01],[2.98155040e-01,4.04124975e-01,6.64084494e-01]],[[-4.68032032e-01,3.13304394e-01,4.09119308e-01],[-8.04559767e-01,4.13860083e-01,-3.41351122e-01],[1.82282317e+00,-9.64638233e-01,6.46531165e-01]],[[4.53156859e-01,-1.33472934e-01,3.26945782e-01],[7.29995131e-01,5.19980311e-01,-6.01284429e-02],[-2.01562464e-01,3.11499149e-01,-2.34218255e-01]],[[5.36239445e-01,-2.04377383e-01,-6.09031282e-02],[-3.19960564e-01,7.75746763e-01,7.21399248e-01],[-1.11360097e+00,9.05544758e-01,-3.06664318e-01]],[[-9.15124416e-01,-2.78578788e-01,1.38600022e-01],[-1.35869730e+00,1.07063882e-01,-6.29513085e-01],[5.38308203e-01,3.14937942e-02,-5.84746301e-01]],[[-3.73227179e-01,-2.42429927e-01,-5.62840700e-03],[3.11864585e-01,-7.32164606e-02,4.81962442e-01],[5.62265441e-02,-1.05083063e-01,1.09025881e-01]],[[-3.49142164e-01,5.07112563e-01,-2.18896195e-02],[1.90452620e-01,6.13167249e-02,3.38817328e-01],[7.66918540e-01,-2.17533469e-01,-8.96900535e-01]],[[-8.44386756e-01,5.39699495e-01,-7.12680593e-02],[-2.07625225e-01,4.07431394e-01,3.65648955e-01],[-3.24371785e-01,3.47184613e-02,-3.28712583e-01]]],[[[-2.89105982e-01,-2.06990615e-01,1.26104519e-01],[-4.07001793e-01,3.62102389e-01,-4.40435670e-02],[9.67053622e-02,-1.26595721e-01,1.38936564e-01]],[[-2.04114974e-01,-4.00650710e-01,-8.72653544e-01],[-4.12567973e-01,9.84706953e-02,-7.86093950e-01],[-2.27523416e-01,-2.62094140e-01,-2.70317644e-02]],[[3.76428962e-01,-9.00887921e-02,-9.00158763e-01],[-1.26562536e-01,5.72767220e-02,1.61113352e-01],[-5.01175821e-02,-1.06148370e-01,4.59794283e-01]],[[2.55950898e-01,-5.68889320e-01,9.46616888e-01],[-1.72254280e-04,-2.02137046e-02,-1.62091777e-01],[2.94475347e-01,3.67245346e-01,-2.24789038e-01]],[[1.96041256e-01,5.76048315e-01,-7.65747502e-02],[2.29308568e-02,4.59420197e-02,-1.00862801e+00],[2.37673260e-02,-4.29072499e-01,9.09091234e-01]],[[-1.69632778e-01,-5.18765390e-01,-3.06529030e-02],[6.03151955e-02,1.38323948e-01,-5.65221719e-03],[2.63360795e-02,-2.70479918e-01,3.87301117e-01]],[[-2.00515077e-01,2.69328564e-01,3.90328145e+00],[-8.22723731e-02,2.13739634e-01,-5.17500222e-01],[-2.15906233e-01,-2.62763172e-01,-1.68887496e-01]],[[2.63894826e-01,-2.58437514e-01,-2.71150500e-01],[-4.44526449e-02,-4.27048445e-01,5.98706976e-02],[-2.87389802e-03,4.86766756e-01,3.68300267e-02]],[[3.50246757e-01,5.12094259e-01,-2.47571445e+00],[-1.71185181e-01,-8.03539336e-01,-1.43794388e-01],[1.05898000e-01,-7.80725420e-01,3.84418759e-03]],[[1.49187088e-01,1.98059574e-01,-4.61363405e-01],[-4.38495399e-03,3.28048706e-01,-1.01777531e-01],[4.13115323e-01,1.01936854e-01,-2.76991054e-02]],[[2.20795020e-01,1.17158115e-01,5.10805786e-01],[2.35595964e-02,3.47442091e-01,2.25813597e-01],[6.56607375e-03,1.54556269e-02,1.29709989e-01]],[[2.63658524e-01,2.20593035e-01,2.33966446e+00],[-2.09778979e-01,8.71427730e-02,8.69197547e-01],[2.68272549e-01,-3.32349479e-01,-7.81036913e-02]],[[1.72440812e-01,4.91066277e-01,1.49762189e+00],[2.89642997e-02,-2.22957969e-01,-1.56818613e-01],[4.00620252e-01,-2.33328283e-01,2.39424855e-01]],[[-1.64329559e-02,2.81768292e-01,5.68612456e-01],[1.76651552e-02,2.87464172e-01,-6.09574378e-01],[-5.17217934e-01,-6.28937483e-01,-3.27387840e-01]],[[-2.32015267e-01,3.05290401e-01,1.46771538e+00],[-2.17457786e-01,4.21937779e-02,2.17137441e-01],[1.34050503e-01,-7.95577988e-02,1.38240168e-02]],[[1.06132939e-01,3.41863960e-01,1.02146935e+00],[4.95049179e-01,-1.75094455e-02,2.54318446e-01],[1.05776275e-02,1.47840127e-01,-5.44732332e-01]],[[-2.29206279e-01,-2.28837475e-01,-1.83358502e+00],[4.85392749e-01,-1.05972998e-01,-1.14648109e-02],[2.14043006e-01,-4.50986717e-03,9.69495624e-02]],[[1.07683077e-01,1.73944265e-01,2.60115147e-01],[-1.83383763e-01,1.04906119e-01,8.68735984e-02],[-3.92300963e-01,2.78506219e-01,1.21691808e-01]],[[-4.26989466e-01,1.38074756e-01,5.62269211e-01],[-1.62506402e-01,5.01670063e-01,2.22524226e-01],[-1.77955016e-01,-1.38895738e+00,1.81325650e+00]],[[1.43192053e-01,4.18012112e-01,-2.33297661e-01],[-2.10878655e-01,-7.85508305e-02,4.87888828e-02],[-4.17721868e-01,-7.93891609e-01,7.53850400e-01]]],[[[0.7100183,-0.12704374,0.02640769],[0.0582644,0.6134307,-0.0678644],[-0.54017615,0.0960613,-0.67547864]],[[-0.31070334,-0.14437945,0.5339351],[1.019623,0.30912662,-0.33967182],[-0.1641311,-0.1462753,0.78711474]],[[0.3007988,0.16489786,-0.6721967],[-0.04713091,-0.07935885,-0.2764025],[-0.638286,0.39946416,0.02591014]],[[-0.35614115,-0.31268883,0.9813876],[0.36337793,-0.33312988,-0.45571515],[0.7446976,-0.6552217,0.3529086]],[[-0.616807,0.23177697,-0.17393447],[0.29946062,0.20192441,0.43279696],[-0.42396075,-0.17782813,0.76682013]],[[-0.15537596,0.23475626,0.3397343],[-0.2786344,-0.30674538,-0.27959853],[-0.03071366,-0.34894377,-0.18225142]],[[0.25626108,-0.9590114,-0.20659009],[-0.11396067,-0.5707966,0.09647206],[-0.14909312,-0.124829,0.8720171]],[[-0.6813214,-0.06174439,0.60536945],[-0.02049182,-0.05298461,0.19711138],[-0.3355912,0.84192723,-0.36877573]],[[0.16220734,-1.006953,-0.1730119],[0.76465654,-0.22186126,0.5928418],[-0.16767985,0.5004856,-0.57417923]],[[-0.3124351,-0.28005967,0.12393948],[0.14562304,-0.6593056,-0.33558404],[0.2389389,-0.6833428,-0.26118922]],[[-0.42560288,-0.06994176,-0.946744],[-0.6180383,-0.5869326,0.11806546],[-0.08270666,0.02624529,-0.16171479]],[[0.2913849,-0.500485,0.61903024],[0.15145665,0.26527596,0.06760554],[0.3501567,-0.13395157,0.42036337]],[[0.33115187,-0.29368964,0.72554874],[-0.6846021,0.45841277,0.89870703],[-0.43227935,-0.43996912,-1.2260299]],[[-0.46236232,-0.02600071,0.5728605],[0.43006197,-0.40083507,-0.10673892],[-0.64596176,1.1649922,0.5731744]],[[0.03016742,0.30711946,-0.24745616],[0.12515648,-0.12533723,-0.2750563],[-1.1546512,-0.37436146,0.5449087]],[[0.42573562,-0.15258676,0.24649213],[-0.22508061,-0.6919214,1.1328915],[1.4940506,0.34120044,0.24205023]],[[0.04511696,0.24391684,-0.14240187],[-0.4597921,-0.6680155,-0.4772734],[0.77195626,0.41156894,0.09986157]],[[-0.0117895,0.05741184,0.24946253],[-0.17105982,0.40761855,-0.20972562],[-0.6707646,0.2444475,0.35531628]],[[0.4141002,-0.19832766,0.87654376],[0.2108185,0.6844834,-0.20316613],[-0.3738268,1.2788651,0.16042]],[[0.35537004,0.19513124,0.05157936],[-0.24215457,0.07737827,0.5326761],[0.8129398,0.71476525,-0.20796435]]],[[[-1.026442,-0.05230237,-0.3678127],[-0.17896433,0.26344156,0.21714537],[-0.39448583,0.00774003,-1.6168114]],[[-0.22725977,0.0383399,-0.4573793],[0.23834582,0.32185632,0.57316136],[-0.1346152,-0.09206164,-0.617558]],[[0.36356848,0.2244791,0.1579794],[-0.00516459,0.11117031,0.21876493],[0.5359809,0.6408679,-1.9646636]],[[-0.66693443,-0.52144605,-1.2848866],[0.05799587,0.09963461,0.38528636],[0.18224762,0.34301737,-0.62438107]],[[0.4847311,-0.03375922,0.22783267],[0.27546987,0.18226354,0.1128625],[-0.48324397,-0.16362259,0.0372135]],[[0.10671591,0.36450106,0.5171798],[-0.09710934,-0.02083975,-0.09261373],[-0.73410153,0.7062345,0.2362949]],[[-0.07757923,-0.12577198,0.53567696],[-0.09090322,-0.09283787,-0.27308628],[0.17756712,0.33319774,0.25945038]],[[-0.32564697,-0.11050455,-0.19307931],[0.07616641,0.05607222,0.28943524],[0.64212257,0.43309098,1.6173427]],[[-0.01809475,0.3284086,0.5798146],[-0.4085248,-0.01428501,-0.47859272],[-0.02023879,-0.13883278,-0.8261009]],[[0.21845219,-0.0426852,-0.57085747],[0.08305979,-0.00568595,0.0946573],[-0.5846616,0.0384199,-0.5176782]],[[0.801765,-0.00885725,0.40144163],[0.19571303,-0.17943832,0.01676759],[0.5960422,-0.16792777,2.0931802]],[[-0.43545896,-0.1581844,-0.7027747],[0.44510713,-0.54977477,0.03897255],[0.30407944,0.7345537,-1.2955048]],[[-0.67857504,-0.04547599,-0.04980535],[-0.42916858,-0.25673875,0.10869587],[-0.24549116,-0.45854947,0.2492114]],[[-0.38609302,0.00791149,-0.24048077],[-0.85687697,-0.08327962,0.26463518],[-0.07335228,0.56040055,-0.59273344]],[[-0.11701084,-0.46026883,0.2867179],[-0.13375755,-0.36097562,0.18208626],[0.40649554,-0.4213575,1.2708572]],[[-0.11571209,-0.1287406,-0.14563088],[0.14329624,-0.29326874,-0.00539708],[0.96656626,-0.15594088,0.8456221]],[[0.32168603,0.03935226,0.32919887],[0.21868058,-0.15834594,0.24114993],[0.7737648,0.5012559,1.1401942]],[[-0.54239327,-0.7863644,-0.44553134],[-0.14567046,0.3081747,-0.34099767],[0.32738957,-0.03584274,-0.5980712]],[[-0.3434627,-0.12264763,-0.36420217],[-0.06991916,-0.1291021,0.2966015],[0.2171048,0.19347921,-1.4624922]],[[0.7457832,0.58121246,0.16133784],[0.14529821,0.48639125,-0.4495059],[-0.11320419,-0.5242444,0.1709976]]],[[[0.17121337,0.1582012,0.40088302],[-0.242659,-0.33152574,-0.32762438],[0.135931,-0.70794487,0.46139252]],[[0.5881802,-0.21946286,-0.17051242],[0.7422856,0.0343767,-0.17119752],[-0.2648267,-0.02937636,-0.12195239]],[[-0.26873028,0.33858883,-0.25297934],[-0.44206938,0.5440791,0.0425081],[0.111375,-1.0415802,1.5735415]],[[-0.7680815,-0.01514764,-0.35738227],[0.49749872,0.21575856,-0.20105988],[-0.36031708,-0.13351305,-0.03101336]],[[0.4871632,-0.06652574,-0.9394802],[-0.7221068,-0.68132037,0.0898203],[-0.08259966,1.1251383,0.5826566]],[[-0.12907551,0.51943415,0.095861],[0.2142534,0.06623989,-0.25093552],[-0.46352884,0.09242593,-0.26707086]],[[-0.2930116,0.29109028,-0.02494384],[0.38077563,0.11244847,0.38463807],[-0.16197272,0.9152504,0.01153814]],[[0.03033366,-0.15731068,0.3270961],[-0.01089873,0.6184519,0.41305694],[-0.3291178,-0.12225036,0.56727314]],[[-0.07916821,-0.42665836,-1.1496059],[-0.03579021,0.15057313,-0.35789868],[0.63876617,-0.14734715,-0.72453743]],[[-0.11556543,-0.529358,-0.19350107],[0.13719854,0.18192275,0.5154966],[-1.0721947,-1.0155777,-0.2083679]],[[-0.29513273,0.0040116,0.14897761],[-0.21831177,-0.35931256,0.52177274],[-0.14554997,-0.06552567,-0.8683858]],[[-0.4988684,-0.1286085,0.09439161],[-0.9744444,0.5187874,-0.24597242],[0.17864107,1.3630097,1.514096]],[[-0.16355528,-0.15446085,-0.53924793],[0.4055534,-0.15476331,-0.01298149],[0.2676012,-0.4892281,-0.39438483]],[[-0.02686042,0.6014071,0.19697021],[0.22906376,-0.31185144,0.47552088],[-0.13268308,-0.04542525,0.13612147]],[[0.8538424,0.0693034,0.21964289],[0.45637244,-0.09891893,0.10471503],[0.3201283,-1.415832,-0.9512449]],[[-0.28776222,0.4976021,0.07560616],[0.26464635,1.3894967,0.15027659],[0.5427143,0.22247995,-0.28008077]],[[-0.4175228,0.08024277,0.05501814],[-0.3641858,0.10380868,-0.23553392],[-0.77057844,0.76760054,-1.5758821]],[[-0.15526585,-0.09490582,-0.13239212],[0.0437303,-0.421258,-0.286397],[-0.5649753,0.3020788,1.1882799]],[[-0.19959372,0.12418158,-0.00692328],[-0.2439645,0.14759736,-0.28017357],[0.26430312,-0.5192609,-0.21804279]],[[-0.01773276,0.52157617,0.00640317],[0.13873042,0.06111957,0.2956765],[0.00981526,0.32063273,-0.2279537]]],[[[-0.16065933,0.09029232,-0.02946904],[0.0732988,0.52221495,-0.13359241],[0.03158408,0.3290652,-0.02863527]],[[0.12618199,-1.4491285,0.05797429],[-0.12746383,0.07529892,0.1602091],[-0.7359247,-0.3167436,-0.5633049]],[[0.20819032,-0.01196999,0.64023095],[-0.916591,0.8427052,0.23617034],[0.640111,-0.7841738,0.3681866]],[[0.49065918,-0.903802,-0.07541297],[-0.22766115,0.10944135,0.01413125],[0.67131877,-0.28632477,-0.628911]],[[-0.8834865,0.459807,-0.2601502],[-0.08981187,-0.45680237,-0.38047826],[-0.2691071,1.2637659,0.620134]],[[0.4961243,-0.93150944,0.617867],[-1.0044444,0.34284076,-0.27074367],[0.10022005,0.30427656,0.27730197]],[[0.2290788,0.20009574,0.71439105],[-0.43374404,0.46653262,-0.7746694],[0.87759775,0.02811954,-0.98028797]],[[-0.29862267,-0.6183967,0.47116557],[0.70593834,-0.6213463,0.29737407],[-0.76993066,0.9402925,-0.7018323]],[[0.07682166,0.99388486,-0.6254754],[0.6342901,-0.1985751,-0.5176501],[0.429753,0.08735985,-1.0024837]],[[0.6364033,0.19077212,-0.6024538],[-0.14815049,0.3422436,0.18163216],[-0.24718246,0.12461628,0.6790116]],[[-0.3546164,0.37367398,-0.28458816],[0.676572,0.09889109,-0.01149668],[0.00784031,-0.15869091,-0.46266666]],[[0.039857,-0.4458923,0.40137428],[-0.26889732,-0.31279418,0.43951222],[-0.4238175,-1.1387665,-0.11974192]],[[-0.11169428,-0.15193127,-1.2554625],[0.91332674,0.3493832,-0.10790989],[-0.2566732,-0.66693217,0.19666184]],[[-0.0773631,0.01062035,0.7574194],[0.12037458,-0.12267352,0.02627774],[-0.36286175,0.41103035,-1.9778273]],[[-0.2361807,-0.4578037,0.60440683],[-0.33794495,-0.2119174,-0.49892595],[-0.08302987,0.37217548,-0.01775203]],[[0.13655247,-0.07663216,0.02589598],[-0.26949632,0.42329347,-0.28878474],[0.9722963,-0.5945996,1.2392825]],[[-0.95816195,0.56932306,-0.12965201],[-0.31581777,-0.51200646,0.46678254],[0.3007543,0.4528188,0.12877207]],[[-0.11449932,0.39634264,-0.2336167],[-0.22816662,-0.36846864,0.03447097],[-0.5285636,-0.40565085,-1.0153674]],[[0.22250417,-0.38141066,0.38191733],[-1.1582626,-0.04619879,0.5787137],[0.3495345,-0.53698045,0.01586232]],[[-0.4900215,0.2054865,-0.33807597],[-0.3403342,-0.21829921,0.24090059],[0.06882469,0.23233974,0.79468805]]],[[[0.72986096,-0.20253101,0.6525715],[0.07272762,0.5298863,-0.681121],[0.97514486,-0.35309047,-0.33368665]],[[1.0667821,0.11669867,0.72254497],[0.5820941,0.05447967,-0.3794789],[-0.03530502,-0.34662953,-0.35661915]],[[-0.51476324,0.04951719,-0.68708646],[-0.7082952,-0.05230324,0.7783933],[-0.29502583,0.08406062,-1.230273]],[[0.05963232,0.3517481,1.187795],[0.26597038,0.00332743,-0.4891978],[0.08255611,-0.5601425,-0.34059626]],[[-0.29768008,-0.02749136,-0.9618959],[-0.1993402,-0.05035613,0.1869204],[-0.1483351,0.23768452,0.38146898]],[[-0.9851282,0.08266594,-0.08416374],[-0.3909846,-0.05674564,-0.27044958],[-0.70333505,0.21759258,-0.7730733]],[[-0.9665962,0.8139456,0.22157863],[-0.00716298,0.17898391,-0.14133832],[-0.21623899,-0.0924494,0.24090481]],[[0.62856287,0.05059973,0.14897062],[-0.45315516,-0.01609755,0.0090593],[-0.39161897,0.2164022,-0.35841897]],[[0.48145366,0.4115062,-0.44416922],[0.33872262,0.30974486,-0.05616322],[-0.6283265,0.03883355,-0.12791336]],[[-0.5398567,0.09327692,-0.09652771],[-0.43994358,-0.01904628,-0.48214966],[-0.6345438,-0.5131242,-0.4125772]],[[-0.85898393,-0.1053607,-0.10600242],[0.07452881,-0.03046198,-0.05788572],[-0.68209827,-0.21606107,0.37802657]],[[0.24274181,0.22162801,0.88003945],[-0.04387147,0.34532657,0.6025196],[0.43817633,0.44021723,-0.9363201]],[[1.0209303,-0.52758515,0.61402106],[0.09207951,0.38930142,-0.45462224],[0.10274735,0.34702644,-0.1921488]],[[0.90556675,-0.01213234,0.93029326],[0.06367703,0.64953476,0.05393686],[0.04511961,-0.09935246,0.7705762]],[[0.6793364,0.4007303,0.41412497],[0.155529,-0.22288112,-0.32446587],[-0.32619438,-0.26275823,0.26175454]],[[-0.91034347,0.43326986,0.1577018],[-0.2702349,0.4333001,0.328146],[0.9075903,0.38824698,0.20172034]],[[-1.0441083,0.24878289,0.22989435],[-0.5070619,0.14837757,-0.03903108],[-0.18885207,-0.29958585,0.5516566]],[[1.3779563,-0.04018363,0.2981623],[0.73368853,-0.06895801,-0.17982493],[0.47419867,-0.15872869,-0.83228487]],[[-0.18129833,0.20320804,0.25505865],[0.4247328,-0.08552583,-0.18088219],[-0.7257271,1.2734721,-0.37731436]],[[-1.124071,0.41893038,-0.33516416],[0.5195971,0.14539456,-1.1174359],[-0.9393404,-0.14714743,0.37501794]]],[[[-0.16169688,0.58707607,0.43624145],[-0.17278802,-0.32722068,0.21898676],[1.5458809,-0.23019643,-0.26303723]],[[0.4317174,0.6354158,0.12837999],[0.21601857,-0.50088334,0.0077294],[-0.52283454,0.3150104,-0.76595634]],[[-0.11606688,0.15367715,-0.3288874],[-0.38766325,-0.39208138,-0.23816699],[0.22165105,-0.87381876,0.28318]],[[-0.11454644,-0.03439311,1.0396165],[0.01604826,0.01618293,-0.43821877],[-0.06231742,-0.05307898,0.30624455]],[[0.43317497,-0.6463441,0.42771474],[0.43404552,-0.01861738,0.0729782],[-0.75809497,-0.4288469,-0.04448274]],[[-0.10330034,0.14169466,-0.3853346],[0.32149854,0.05017285,-0.10226671],[-1.8199152,-0.2569812,0.16457161]],[[-0.09679028,0.36126947,-0.30206576],[0.2274635,-0.4333734,0.9409425],[0.35696492,0.3342123,0.0314862]],[[-0.70972526,-0.1060744,1.2253428],[-0.58925855,0.50637573,0.3812849],[-0.9452184,0.04808209,0.6788099]],[[0.36026198,0.02381633,0.37447515],[-0.12465702,-0.24747294,0.37322414],[0.6192712,-0.15061958,-0.78904384]],[[-0.2838742,0.06484602,-0.3228248],[0.43226105,-0.8886948,0.48990566],[-1.6151427,-0.2054093,-0.17510849]],[[-0.27306265,-0.22902992,-0.82173645],[-0.45508322,-0.22483182,-0.14858767],[-0.7710585,0.10893402,-0.4950868]],[[0.07347644,0.573817,-0.41184792],[0.08240751,-0.14430632,0.3673869],[0.67043877,-0.30448937,1.393543]],[[0.607187,-0.39849707,1.0499291],[-0.07117452,0.9179402,-0.164187],[0.48718432,0.3663918,-0.0955264]],[[0.6455796,0.44908974,0.96764445],[0.49034107,-0.00459697,0.0805207],[0.7062231,-0.05793617,-0.11131828]],[[-0.07605108,0.71212196,-0.6595551],[0.1616749,-0.18848047,-0.00936012],[-0.06058409,0.11270902,-0.06365563]],[[0.4860215,0.0336262,-0.21776368],[-0.22557525,0.24149598,-0.52333933],[-0.759492,-0.8967438,0.03103926]],[[-0.2197258,-0.08866726,-1.0806384],[0.2190117,0.36945653,0.25320336],[-1.0307355,-0.26072976,0.22661135]],[[0.11158783,0.16874725,0.02589671],[-0.0759662,-0.12579697,0.06100015],[0.3067051,0.54010606,0.906842]],[[-0.06717593,0.4515033,0.04984479],[0.92022115,0.33038294,-0.50512487],[0.23835231,0.20271093,-0.4370621]],[[0.05992972,-0.22658625,-0.60434353],[0.08079407,-0.0536806,-0.73217404],[-0.42742598,-1.1855992,-0.29530638]]],[[[8.49572301e-01,3.26020926e-01,3.13437849e-01],[2.42842242e-01,1.90829784e-01,-4.97270823e-01],[-2.19833359e-01,-4.73686486e-01,2.52972126e-01]],[[8.33194796e-03,-1.40887558e-01,-1.61826104e-01],[5.09508908e-01,-1.20305955e-01,-1.23059481e-01],[-2.73459792e-01,9.56659615e-01,3.05653811e-02]],[[7.72528052e-02,1.98005643e-02,-1.35213554e-01],[-2.25802571e-01,-3.58638465e-02,8.19924474e-02],[1.08254659e+00,5.20761348e-02,7.49723494e-01]],[[-3.02695930e-01,-6.91083670e-02,1.30464390e-01],[-1.47337862e-03,-3.68878357e-02,2.67652273e-02],[-9.36517194e-02,-2.20750377e-01,-1.03213266e-01]],[[-9.38367248e-01,-2.29218245e-01,-4.21263963e-01],[-5.99002361e-01,-3.54572805e-03,3.99953514e-01],[-9.99862671e-01,1.20919359e+00,-9.61078405e-02]],[[2.90724635e-03,4.28763330e-02,1.77180722e-01],[1.04338539e+00,4.50441599e-01,-1.42126959e-02],[-1.77375174e+00,3.90592101e-03,-8.81946981e-01]],[[1.59885466e-01,1.93324819e-01,6.41744852e-01],[7.56976604e-02,8.28177631e-02,-5.48156202e-02],[-1.13292229e+00,2.68784076e-01,4.54533957e-02]],[[3.10835332e-01,-6.77516237e-02,-1.16800904e-01],[-2.22373605e-01,-2.57252663e-01,1.69659227e-01],[1.11280751e+00,5.98079383e-01,-6.68954074e-01]],[[-6.53398991e-01,-1.94363222e-01,-1.16727591e-01],[2.74302363e-01,-3.50193232e-01,-1.58727601e-01],[1.38918497e-02,-6.55940771e-01,7.70299435e-01]],[[-3.45728874e-01,7.00822920e-02,-1.81565687e-01],[1.10812262e-01,-1.73520267e-01,-1.57361224e-01],[-2.67910123e+00,2.17350632e-01,4.71405312e-02]],[[-7.90887773e-01,-1.20022506e-01,1.17688023e-01],[-4.04050082e-01,-2.97561824e-01,-3.93801749e-01],[2.21475720e-01,-3.47857237e-01,-4.39698905e-01]],[[5.74779451e-01,5.80137828e-03,9.29085463e-02],[-1.79653257e-01,-2.03019530e-01,2.05978844e-02],[1.18668771e+00,5.68535030e-01,3.59530807e-01]],[[1.21960685e-01,-2.20115677e-01,8.56844425e-01],[1.60618760e-02,1.21211421e+00,2.35672683e-01],[5.29558182e-01,-9.52448666e-01,2.34024316e-01]],[[1.18594694e+00,4.69738571e-03,-2.39310056e-01],[-4.56979096e-01,-2.21714407e-01,7.04469085e-02],[6.67821109e-01,4.77789938e-02,2.00731382e-01]],[[9.27949846e-01,2.47604325e-01,-2.90111363e-01],[-5.91367744e-02,-4.94844653e-02,-1.75125524e-01],[9.61046517e-01,-6.11595139e-02,1.17317408e-01]],[[2.32296690e-01,5.51525712e-01,-1.01662293e-01],[5.80752075e-01,1.33124823e-02,4.79636967e-01],[-1.84861696e+00,-9.89715934e-01,4.99765635e-01]],[[-3.76887798e-01,-2.35969499e-01,-3.11103016e-02],[-2.22261593e-01,9.90998372e-03,1.83566153e-01],[-1.53478849e+00,-7.57677704e-02,-3.09321791e-01]],[[-6.74650520e-02,-1.51914418e-01,9.69409123e-02],[-3.83490250e-02,-3.48115593e-01,9.17221904e-02],[1.61560512e+00,4.27695543e-01,5.89748263e-01]],[[2.19621301e-01,-3.19407769e-02,2.23372728e-01],[2.92102635e-01,1.77661121e-01,1.86427921e-01],[5.58474138e-02,6.45560503e-01,5.47179461e-01]],[[1.16667621e-01,4.68601704e-01,-3.68111998e-01],[-2.65521798e-02,-5.57258651e-02,1.32925913e-01],[-1.19082952e+00,7.14053214e-01,-2.73067653e-01]]],[[[-8.98634270e-02,7.91565001e-01,2.04781309e-01],[-1.23799279e-01,-1.04056783e-02,4.19244736e-01],[-2.44672745e-01,-4.30264026e-02,1.39189899e-01]],[[1.48033708e-01,-1.89037365e-03,-6.49282873e-01],[-1.82815418e-01,4.74475771e-01,8.61163914e-01],[-2.94445783e-01,9.81395543e-02,-7.67083943e-01]],[[3.16828370e-01,5.00012517e-01,-7.79769197e-02],[-1.26559496e-01,9.33067143e-01,-2.07393467e-01],[-4.26326804e-02,5.25383353e-01,-1.08519089e+00]],[[-2.66673923e-01,3.05312932e-01,-2.40653992e+00],[-1.36411086e-01,-3.81681114e-01,3.31907600e-01],[-2.23458171e-01,4.87469435e-02,-1.70303214e+00]],[[2.72777736e-01,-1.00565843e-01,7.45665610e-01],[2.82299012e-01,3.62258047e-01,5.46274364e-01],[-4.63768572e-01,2.48030245e-01,7.45615065e-01]],[[-3.74081761e-01,-1.06283881e-01,-1.31106690e-01],[8.64125729e-01,5.55767953e-01,-2.27684423e-01],[-9.35467601e-01,-4.66313213e-01,-4.82342392e-01]],[[-2.54991800e-01,-1.21616326e-01,-1.90581396e-01],[-1.67849176e-02,-5.79345465e-01,-1.69333965e-01],[-3.30550730e-01,4.33833539e-01,1.14448078e-01]],[[-8.25271308e-02,-2.52631992e-01,-9.18137655e-02],[4.03699309e-01,-5.73922575e-01,-1.95237741e-01],[2.96914309e-01,8.67383629e-02,-3.81360531e-01]],[[-1.22293368e-01,1.09464131e-01,1.04204953e-01],[2.81500846e-01,2.42481023e-01,2.93957889e-02],[4.70446348e-01,8.90335143e-01,-8.67345035e-01]],[[-8.94745737e-02,3.41679096e-01,1.31819606e-01],[-1.17928023e-02,-3.08547109e-01,-6.76962733e-01],[-2.61879444e-01,-4.90528882e-01,3.70768681e-02]],[[-6.49386346e-01,-3.34166169e-01,5.11867702e-01],[-1.38852730e-01,-6.69548094e-01,2.24979788e-01],[7.70177901e-01,-2.07591560e-02,4.03917581e-01]],[[3.44676912e-01,1.51060671e-01,-3.98004621e-01],[-3.83530259e-02,1.17913283e-01,-7.87722707e-01],[-3.70607018e-01,1.79907486e-01,-1.26477039e+00]],[[-5.73221743e-01,-4.20640349e-01,-5.39321780e-01],[-1.47718623e-01,5.77277970e-03,3.03380340e-01],[8.72651875e-01,-6.97090805e-01,-1.16795504e+00]],[[1.20101348e-01,-3.04266401e-02,-5.83788812e-01],[5.10996580e-01,4.88100886e-01,3.63487691e-01],[7.85311997e-01,7.63711870e-01,5.38123608e-01]],[[7.57504761e-01,-8.66342410e-02,4.38807428e-01],[5.97436875e-02,-5.99589467e-01,8.37627709e-01],[5.11790998e-02,-5.86057782e-01,8.19438398e-01]],[[1.17050372e-01,6.90835238e-01,-1.77809522e-01],[2.31678281e-02,-1.04333453e-01,2.00953707e-01],[2.08185807e-01,8.78699303e-01,5.40515602e-01]],[[6.91790506e-02,3.04617465e-01,8.96233082e-01],[1.21937163e-01,-5.75652838e-01,7.36862049e-02],[-8.75696659e-01,8.03451896e-01,4.53388751e-01]],[[2.00070262e-01,-5.72658360e-01,9.21250358e-02],[2.30650753e-01,-7.00317994e-02,-1.58859089e-01],[-2.17213735e-01,-8.57067704e-02,-7.36274600e-01]],[[-5.91742814e-01,-1.11670591e-01,-1.32963173e-02],[1.09152414e-01,4.55180183e-02,-2.33424395e-01],[-6.82283878e-01,9.66504872e-01,-1.16727732e-01]],[[-3.22778940e-01,1.16631162e+00,2.23889232e-01],[3.44680488e-01,-2.71805614e-01,-2.44091779e-01],[1.13871180e-01,3.94834131e-01,7.04545915e-01]]],[[[0.00738238,0.57213145,-0.44415092],[0.49118665,-0.01737477,-0.363972],[-0.92448527,-0.42378214,0.34836116]],[[-0.05225849,-0.20883933,-0.3078763],[-0.08734306,-0.4178923,0.05749792],[0.45122075,0.5771843,-0.22801173]],[[0.30933052,0.21906146,-0.3853904],[0.32960063,0.04679675,-0.32948226],[-0.5006283,0.38524184,0.07238679]],[[-1.1218299,-0.11052647,-0.68327975],[0.18372184,0.3580258,0.08923346],[0.3180118,-0.04252131,0.17177811]],[[-0.08023597,-0.5394083,0.07605845],[0.29842666,-0.29345983,-0.33003113],[-0.69337475,-1.0962352,-1.1997324]],[[0.6240526,1.0274807,0.02465961],[-0.49695906,0.3001686,-0.18089052],[0.9688291,0.77399033,-0.05857326]],[[-0.76940507,0.22100654,0.9956672],[0.02643316,-0.6854669,0.55065894],[0.17202567,0.5544473,0.16751365]],[[-1.7620075,0.35199153,-0.10924855],[-0.01573423,-0.3987847,0.19624211],[-0.10691389,-0.417463,0.3912979]],[[0.6404208,-0.3007012,-1.6854805],[-0.0622333,-0.20511933,0.18161438],[-0.600549,-0.21266134,1.1006689]],[[0.7377246,-1.0552063,-0.65688795],[-0.43411094,0.13870099,-0.31870624],[0.38601705,-0.4713394,0.18678446]],[[0.5400274,-0.33827204,0.88676053],[0.14639093,-0.96825993,-0.15808037],[0.15614744,0.4545026,0.17484339]],[[-0.66982687,0.08084983,0.6935612],[-0.0942889,-0.34491616,0.09568709],[-0.67048794,0.2931322,-0.11657968]],[[-0.35833025,-0.44853434,0.37602657],[0.23286052,0.79059494,-0.18952318],[-0.3172147,-0.31375208,0.35705674]],[[-0.33720586,0.44535825,0.6643177],[0.26055363,-0.42200416,0.35050797],[0.11520792,1.3628477,0.89896727]],[[0.28559023,0.33521917,1.592654],[0.07748251,0.32615027,-0.28568372],[0.72190595,-0.12676223,0.37552017]],[[-0.15049067,-0.20263584,-0.13658315],[-0.2983527,0.26556838,-0.09999145],[-0.76329935,0.25968194,0.38705546]],[[0.9779449,-0.25025502,-0.9173995],[0.00975807,-0.17466445,0.13448776],[0.14620909,-0.03727596,-0.17774798]],[[-0.4136363,-0.7641283,-0.07788611],[-0.09124999,0.11362112,0.23617125],[0.37659428,0.02439875,-0.58208466]],[[0.00201077,-1.6642833,0.00776348],[-0.07683759,0.17197584,0.07040395],[0.02806846,0.68549895,-0.29328015]],[[0.2861953,0.87274784,-0.3110289],[-0.21455948,-0.20949832,0.15004145],[-0.42930552,0.717463,-0.5775052]]],[[[-0.5001658,0.22058012,-0.25266367],[1.0566765,-0.392889,1.0836571],[-0.16523115,0.2019706,-0.9122172]],[[-0.71889377,0.6269737,0.3399306],[0.7228895,-0.63442504,-0.12705247],[0.09087496,0.40936574,1.0957375]],[[-0.5175502,-0.36908972,0.3598552],[0.3477808,0.35332,-0.13341619],[-0.29518682,-1.752844,1.5096009]],[[0.0488196,-0.07185925,-0.59539515],[0.04152122,-0.03722224,0.15610367],[-0.3281295,0.32070756,0.60028166]],[[0.08764375,0.3230065,0.92392343],[0.82762104,-0.1826195,-0.6351434],[0.948543,-0.7334878,-0.88414073]],[[0.01970087,-0.26343662,0.15009987],[0.26980454,-0.1705821,0.0877162],[0.72670215,-0.4299567,0.60511756]],[[0.5841797,0.2924015,-0.34265026],[-0.23323338,-0.13815774,-0.4523188],[-0.7526881,-0.00207481,0.45057335]],[[0.29696292,-0.4344924,0.04016491],[-1.0379372,0.12172378,-0.58934045],[-0.67099184,0.10366199,1.1686676]],[[-0.1204915,-0.6651581,-0.49160725],[-0.21340203,-0.32926196,0.46015108],[0.0359931,-0.1717029,-1.4259975]],[[-0.37255725,-0.09572598,0.48183763],[-0.46434972,-1.2497178,-0.18912548],[0.38706252,0.5192689,0.48323384]],[[-0.35556108,0.22807477,-0.22364971],[-0.05688492,-0.10108744,-0.08213583],[-0.5230524,-0.13562173,0.75982386]],[[0.03457253,-0.40926456,-0.27842698],[0.49012613,-0.03125056,0.21141073],[-0.828569,-0.48595646,-0.08789965]],[[-0.5311748,-0.23676392,-0.9654952],[-0.6753048,-0.36799285,-0.08114029],[0.09481872,0.25767642,-1.0293106]],[[0.59285384,0.40043435,0.13964707],[0.25703946,0.06092427,0.56355923],[0.46179777,0.00470043,0.4528895]],[[0.57433176,-0.07268868,-0.03013592],[-0.07291684,-0.11275298,0.10106791],[0.49675125,0.38476717,0.48481554]],[[0.39623266,0.01947896,-1.1968787],[0.5311728,0.5434554,0.274916],[0.06495635,0.18167944,-0.46639657]],[[0.11000259,0.06294861,-0.14375946],[0.34558088,0.2966192,0.9727437],[0.15034112,-0.05425832,-0.61045176]],[[-0.52183276,0.00571718,-0.07466206],[-0.34815255,0.44846046,-0.0455948],[-0.33116683,-0.01258484,0.8053043]],[[0.05437224,-0.07155947,-0.19094394],[0.50771725,0.01468676,-0.41968104],[-0.01292724,-0.410163,0.55550444]],[[0.06776655,0.18304655,0.26935565],[-0.25527057,-0.04432065,-0.3266975],[0.88545626,-0.8423188,-0.3717307]]],[[[-6.6664094e-01,2.4916631e-01,-3.2931167e-01],[-4.3488416e-01,-4.9864151e-02,-5.3048342e-02],[-6.2105095e-01,3.2928675e-01,-5.1462358e-01]],[[6.5977359e-01,-9.1391668e-02,-1.7108752e-01],[1.2939021e-01,2.6807690e-01,-6.2140532e-02],[-5.9703994e-01,-5.2716553e-01,-3.6874008e-01]],[[-5.7506019e-01,1.3759524e-01,-1.6182038e-01],[-3.8646597e-01,3.3712059e-01,-5.5564612e-02],[-1.3436887e-01,-5.6986278e-01,-3.0709046e-01]],[[-7.3203459e-02,-1.3882738e-01,6.2353510e-02],[8.4216267e-02,2.7793659e-02,-1.2979205e-01],[5.9464115e-01,6.8135545e-02,6.5869212e-02]],[[9.1312667e-03,2.6612952e-01,5.0746107e-01],[-1.4004281e-01,-1.2102082e-01,1.0261884e-01],[2.4885658e-02,1.0397577e-01,1.0295385e-01]],[[1.6268315e+00,-6.4685747e-02,-7.2410089e-01],[-7.1399522e-01,-7.4426603e-01,-9.6627757e-02],[2.2022383e+00,-3.7004384e-01,2.2607893e-01]],[[2.4840258e-01,2.4723870e-01,-2.7092882e-02],[-1.2250820e+00,1.2119121e-01,1.9933677e-01],[7.4295664e-01,-1.3874876e-01,-1.5836823e-01]],[[8.2311586e-02,-1.6462193e-01,-4.5373879e-02],[5.0098288e-01,-1.8469106e-02,-2.3778386e-01],[-3.3221431e-02,-2.3156238e-01,-7.9012446e-02]],[[-3.0579638e-01,-2.2338778e-01,2.3648658e-01],[1.7266680e-01,4.6497995e-01,3.6647663e-01],[-1.5367715e+00,2.7371284e-01,3.4623101e-01]],[[-7.2114938e-01,-3.1215951e-01,4.1922161e-01],[-4.3684602e-01,-1.2027766e-01,-6.6414520e-02],[7.0340222e-01,-5.9807647e-02,3.4908786e-01]],[[1.3529988e-01,2.1883000e-01,1.3727614e-01],[2.7192459e-01,-2.9570442e-01,3.0749828e-01],[2.9361904e-02,2.8404781e-01,2.2314166e-01]],[[1.5315138e-01,1.5119860e-01,-2.4389686e-01],[5.2435750e-01,-5.5759128e-02,-7.1030796e-02],[-1.0950454e+00,-4.9898982e-01,-4.8633525e-01]],[[-8.4083372e-01,-9.6448749e-02,-3.7427169e-01],[5.8316778e-02,-1.4732639e-01,-2.7629366e-02],[-2.2618291e+00,7.2395867e-01,5.8530204e-02]],[[5.0890772e-04,-7.6996095e-02,-5.1269782e-01],[3.5059011e-01,3.2571915e-01,-2.5294992e-01],[1.3951272e-01,-3.0020216e-02,1.9070604e-01]],[[1.7840172e-01,1.7060056e-02,-3.9152332e-02],[5.3460407e-01,3.7792623e-02,2.0613536e-01],[-1.0822337e+00,-2.3524603e-01,9.2772253e-02]],[[1.6159350e-01,-3.4454346e-01,1.2940014e-02],[-9.4401395e-01,2.2505380e-01,-2.8444013e-01],[8.4711456e-01,-1.3762596e-01,1.3740623e-01]],[[1.8766803e-01,1.6839173e-01,-5.2016538e-01],[-7.9314783e-02,-3.4501565e-01,2.1655516e-01],[2.7109475e+00,-3.1981117e-03,4.4731387e-01]],[[-3.2573506e-01,2.3955481e-01,1.2403811e-02],[1.5233208e-01,1.6754176e-01,5.4736365e-02],[-1.4505219e+00,-3.4935874e-01,-2.7260813e-01]],[[1.0114676e-01,4.7149038e-01,-2.1178854e-01],[-1.7081420e-01,-1.2588337e-02,-6.8230081e-01],[1.0813523e-01,-2.9778701e-01,6.4917964e-01]],[[8.9920622e-01,8.0600500e-02,3.1567103e-01],[2.7057034e-01,1.8513420e-01,-9.8286882e-02],[1.1844370e+00,3.1457755e-01,4.0287519e-01]]],[[[-0.29637522,-0.02689506,0.13340123],[0.73560125,0.01608629,-0.14864412],[0.60203695,-0.33827052,-0.04837879]],[[-0.44187638,-0.40443665,0.21498863],[-0.5595588,0.04285778,-0.37945703],[-0.0324637,-0.4565105,-0.29101798]],[[0.16636118,0.15012124,0.3833448],[0.08709999,-0.46473387,-0.05189573],[0.14829712,0.49484926,-0.23069458]],[[0.32632688,-0.27508393,0.07733962],[0.10115286,0.0054581,0.22832462],[-0.85893095,0.66310287,-0.5731574]],[[-0.4591067,0.34139574,-0.17014758],[-0.67611784,0.64224035,0.48652095],[1.8690708,0.03939111,-0.06715638]],[[0.89683485,-0.50040317,0.16526636],[-0.06733742,-0.02651904,-0.13211672],[0.862058,-0.87653136,0.5086877]],[[-0.40601048,-0.06013231,-0.4180034],[-0.40094352,0.13499254,0.08556195],[0.05755205,0.19972755,-0.16222663]],[[-0.45486835,0.4489216,-0.01781844],[0.44285992,0.3971776,-0.13064301],[-0.52082795,-0.6837268,-0.34252715]],[[0.4799884,0.05531802,-0.06477625],[0.07496376,0.775776,0.63695264],[0.5814799,-0.04823751,-0.11201864]],[[0.34345755,-0.18654338,0.12541595],[0.4076866,0.05149421,0.17791586],[0.52048934,0.11485068,-0.3029172]],[[1.0290661,-0.27761787,0.03167326],[0.06704972,0.00743701,-0.39355356],[-0.05979036,-0.4527357,0.5386685]],[[-0.42209938,0.54798526,0.1573618],[0.23276274,0.01969095,0.6087141],[0.18804108,-0.07492527,0.1879244]],[[0.02557559,-0.10359176,0.01425255],[0.41572583,0.2842616,-1.1396903],[-0.30813104,-0.22254995,0.7092415]],[[-0.8277405,0.34912753,-0.3312894],[0.02464732,-0.12492482,0.16278124],[-0.35088503,-1.7843299,0.75615484]],[[-0.62697965,0.10296901,-0.11965184],[-0.05598208,-0.50467503,0.15140752],[-0.41567618,-0.58360493,0.38239986]],[[0.09875346,0.10752574,-0.31485298],[0.31776518,0.28416836,0.61560416],[-0.8855783,1.8441854,0.00496449]],[[0.795461,-0.11569129,0.10481533],[0.13660982,0.35576984,0.31303242],[0.68015,0.26101857,1.1828555]],[[-1.0421288,0.12087175,-0.16794762],[-0.0222674,-0.01267933,-0.299399],[-1.7248423,-0.05004966,-0.15284897]],[[-0.19799334,-0.3065998,0.26035985],[0.4979662,-0.09619353,0.47522178],[0.42029282,0.356188,-0.38963714]],[[1.5933354,-0.10365754,0.04476176],[0.07665392,0.05847993,-0.11555183],[0.25398424,0.08055246,0.55057424]]],[[[3.23038429e-01,-9.08649340e-02,5.53926267e-02],[1.40756415e-02,-1.81329444e-01,7.52796531e-01],[-7.74572134e-01,-3.22389267e-02,-1.34535158e+00]],[[3.99977684e-01,4.98903096e-01,-8.20803121e-02],[1.51117191e-01,-5.82026362e-01,-5.81039153e-02],[-3.84769261e-01,2.08552424e-02,4.41109762e-02]],[[4.72408086e-01,-8.13884065e-02,-4.35208172e-01],[1.13628387e+00,-6.46448284e-02,1.01223290e+00],[-2.28558630e-01,2.57212281e-01,-2.71573997e+00]],[[1.05464496e-01,5.13311438e-02,-1.33558893e+00],[2.63404101e-01,-1.05652884e-01,2.36590222e-01],[1.88643709e-02,6.77933455e-01,-3.31314914e-02]],[[5.15624881e-02,-2.82672405e-01,9.74832103e-03],[-6.74607679e-02,-1.08911030e-01,-5.43263443e-02],[-2.81971246e-01,-1.27448332e+00,-2.99651653e-01]],[[-3.11769992e-01,1.61207229e-01,5.38624167e-01],[3.82426232e-01,3.90615910e-01,-1.50400847e-01],[4.99057353e-01,3.91272902e-01,-1.92002714e-01]],[[-1.72061324e-01,-6.30538821e-01,-7.06405520e-01],[5.67739129e-01,-1.73588082e-01,2.09558517e-01],[6.85479641e-01,3.08596641e-01,8.97445738e-01]],[[1.26781628e-01,9.05138403e-02,2.10931480e-01],[-1.70509785e-01,1.10763460e-01,-6.32531166e-01],[2.38395054e-02,7.79321045e-02,-1.66577861e-01]],[[3.49214435e-01,3.63545597e-01,6.37500167e-01],[8.22485268e-01,-6.03927649e-04,-6.35161579e-01],[-4.02024090e-01,-4.47806746e-01,-1.22331537e-01]],[[1.79540068e-01,3.53222787e-01,3.09370279e-01],[-1.48490295e-01,-1.03973627e-01,-7.75506198e-02],[6.31074071e-01,1.32313043e-01,-1.21591485e+00]],[[-1.42565683e-01,-3.65583241e-01,4.54334110e-01],[9.93080884e-02,-4.42941844e-01,2.71436721e-02],[4.46052194e-01,3.10400784e-01,1.16982365e+00]],[[3.57436948e-02,-3.62324804e-01,-8.76647472e-01],[4.20784712e-01,6.45741224e-02,9.83744022e-03],[-5.97321153e-01,-2.51939267e-01,-2.18081522e+00]],[[-3.98638904e-01,4.45099592e-01,2.46509500e-02],[-1.63244054e-01,-8.08998272e-02,-2.04002634e-01],[-3.98559362e-01,2.93965898e-02,-4.20810729e-01]],[[2.41614193e-01,2.18102366e-01,-3.12516302e-01],[1.21628195e-02,1.27316758e-01,-2.63099134e-01],[9.60194468e-01,8.82119477e-01,6.10453725e-01]],[[-3.82148445e-01,-5.55299342e-01,1.64252788e-01],[5.81922948e-01,-1.87989205e-01,4.98334229e-01],[1.21159442e-01,-4.49523389e-01,4.17120367e-01]],[[1.61786899e-01,3.41259301e-01,1.71352103e-01],[-1.87468663e-01,-6.32476926e-01,-3.83772135e-01],[4.42423336e-02,2.33245894e-01,-1.21086590e-01]],[[-1.15871113e-02,-1.15106083e-01,7.94412017e-01],[-4.74825442e-01,-3.00987810e-01,-7.69559741e-01],[9.71565902e-01,4.17951137e-01,1.29154253e+00]],[[-1.11127906e-01,2.07026064e-01,-4.99370128e-01],[4.39885736e-01,4.46240097e-01,2.41076335e-01],[-4.28286672e-01,-3.18541378e-01,-9.41766858e-01]],[[-2.59682477e-01,5.21818817e-01,-1.72512725e-01],[-4.51072127e-01,-2.55883306e-01,4.03276175e-01],[-6.63408995e-01,-8.24990511e-01,-5.51903784e-01]],[[-4.60220575e-02,-4.80725169e-01,-4.95815314e-02],[-9.39384624e-02,-1.76390782e-01,-4.48426679e-02],[-2.17263941e-02,-4.88167167e-01,6.58236861e-01]]],[[[3.0017442e-01,-1.8147084e-01,-1.9312219e-01],[1.4607921e-01,-2.3264898e-01,5.8010016e-02],[1.6460030e+00,6.5746659e-01,6.9903344e-01]],[[-3.2163122e-01,2.5204405e-01,-2.8472531e-01],[-2.0368834e-01,-2.7918043e-02,4.8679328e-01],[-2.3015322e-01,-5.6134641e-01,-6.5616660e-02]],[[-8.1385180e-02,2.7896503e-01,2.8474617e-01],[1.0940855e+00,-1.8919902e-01,-3.5173759e-01],[1.7960175e+00,2.0127736e-01,-4.2491391e-01]],[[1.0649989e-01,-5.2999888e-02,-5.8383840e-01],[-3.2776028e-01,-4.7780346e-02,5.0814454e-02],[1.5757671e-02,-6.2137060e-02,-3.4101000e-01]],[[-4.9634138e-01,2.4877001e-01,-4.7886655e-01],[2.5400761e-01,3.8313961e-01,-2.5329390e-01],[-7.5808752e-01,-1.0543245e+00,9.3392617e-01]],[[-3.1762409e-01,3.6286303e-01,3.8387999e-01],[-2.1516515e-01,-4.3466225e-01,2.5209384e-02],[-1.5822197e-01,-7.5882755e-04,3.7096927e-01]],[[-8.9298636e-02,-2.5849003e-01,1.1458618e-01],[-2.0341434e-01,3.1120968e-01,-3.1406245e-01],[-4.5453164e-01,2.9651606e-01,-4.6868429e-02]],[[8.5337240e-01,-1.3826431e-01,-1.3970308e-01],[-3.7956592e-01,4.5704741e-02,-2.6248243e-01],[-4.4858882e-01,-1.3690057e-01,-6.7933440e-01]],[[2.5345835e-01,-4.7461558e-02,2.1544632e-01],[-4.2533171e-01,-8.3401270e-02,-5.7310921e-01],[2.0877950e+00,-2.9346165e-01,-2.1250744e-01]],[[4.3271312e-01,5.5586588e-01,-6.1691803e-01],[5.3540754e-01,-3.7146833e-02,3.8367850e-01],[-9.8242694e-01,7.7066886e-01,-1.7194976e-01]],[[-2.0070776e-01,-2.6932653e-02,3.6867502e-01],[-1.2157434e-01,2.9235241e-01,1.6559917e-01],[1.9304696e-01,-6.5989119e-01,-6.7882888e-02]],[[-2.1420553e-01,-3.8934830e-01,5.6784666e-01],[2.8013632e-01,5.6055490e-02,-2.1251149e-01],[8.8228619e-01,-1.0582801e-01,-1.8541221e-01]],[[1.0924920e+00,1.1866848e-01,2.0647977e-01],[-4.5190853e-01,-5.3875071e-01,9.4463341e-02],[2.1557975e+00,-1.5461867e-01,2.0120595e-01]],[[-4.1296098e-01,5.1000607e-03,-2.3779696e-01],[-5.4831219e-01,5.9616846e-01,3.4793773e-01],[-4.7235045e-01,2.6122048e-01,4.8615959e-01]],[[-2.5928821e-02,-5.0952464e-01,-4.3503538e-01],[-3.5210931e-01,1.0515476e-01,1.0432899e-01],[1.2518619e+00,-2.6249033e-01,3.0165234e-01]],[[-3.3144483e-01,1.4643957e-01,4.9195167e-01],[2.2896202e-01,-2.6053464e-01,-3.4817642e-01],[-1.4304821e+00,7.6394016e-01,-4.6555939e-01]],[[-1.5522230e-01,2.1900536e-01,2.9270741e-01],[-4.1919285e-01,2.2545567e-01,-5.2316374e-01],[-1.1294292e+00,-2.8453323e-01,8.1668697e-02]],[[1.0614961e+00,-3.8438362e-01,-5.0842488e-01],[-2.7626640e-01,5.7263888e-02,-1.0873527e-03],[9.7695833e-01,-3.2655466e-01,-4.6152356e-01]],[[3.1668803e-01,1.1981206e-01,-1.3737717e-01],[-4.7369707e-01,1.5924563e-01,4.0969023e-01],[9.0154076e-01,-4.4273180e-03,1.2569974e-02]],[[-4.5656756e-01,-3.1602165e-01,-4.7191364e-01],[-1.5862514e-01,3.9178777e-01,-1.3985533e-01],[-8.9627862e-01,-2.6049402e-01,5.7047838e-01]]],[[[-4.8333240e-01,1.0173729e-01,-2.2820103e-01],[1.2335312e-01,7.8679711e-01,-1.3508414e-01],[-6.1704063e-01,4.7859752e-01,-8.7159038e-01]],[[-6.3803768e-01,-1.9115618e-01,4.9734619e-01],[-1.3390896e-01,3.7504706e-01,3.5731453e-01],[3.0357847e-01,7.6689893e-01,-2.0260154e-01]],[[-2.1586175e-01,-3.2443473e-01,3.2812482e-01],[1.2682560e-01,4.5876101e-02,-3.3108395e-01],[-7.6647931e-01,6.7763370e-01,-1.0847485e+00]],[[-3.0515274e-01,-6.6836143e-01,-1.0134635e+00],[8.3241634e-02,3.0311570e-01,5.9911048e-01],[2.2723757e-01,2.2440146e-01,-1.0867749e+00]],[[7.9389733e-01,1.3127299e-01,9.4065470e-01],[1.5341313e-01,5.6205902e-02,4.0080500e-01],[-6.2853676e-01,-6.2961861e-02,1.0990361e+00]],[[-8.9518949e-02,7.4125908e-02,6.4029157e-01],[-3.4882760e-01,-9.5245570e-02,-1.9497316e-01],[1.1642134e+00,-5.7127875e-01,-7.2798423e-02]],[[-2.6143607e-01,-1.3622914e-01,-1.6479491e-01],[2.1714298e-01,2.5745347e-01,-1.6758215e-01],[9.0051967e-01,-2.0830739e-01,-7.8550494e-01]],[[4.9635714e-01,-2.3252107e-02,6.9806546e-02],[-1.3764581e-01,5.4277480e-01,-3.4719935e-01],[-3.4531096e-01,4.1491261e-01,-2.9758132e-01]],[[5.8350369e-02,5.4762906e-01,9.5758080e-01],[-3.4012827e-01,5.0114924e-01,5.2318327e-02],[-2.8254372e-01,9.0999049e-01,-6.0454887e-01]],[[-7.6561791e-01,-3.9372963e-01,-6.2035412e-01],[-2.1141012e-01,-3.1767488e-02,1.5024300e-01],[6.0876852e-01,3.1335399e-01,-1.8013665e-01]],[[-4.1831282e-01,-1.7688012e-01,-1.7894223e-02],[-3.0689955e-01,-5.4585058e-02,1.8011808e-01],[5.2613869e-02,3.4807363e-01,7.9537791e-01]],[[-4.8648837e-01,-5.4303821e-02,-1.2488303e-03],[-7.0185876e-01,-1.1807911e-01,-6.7895579e-01],[-2.1389849e-02,1.1593665e-01,-1.2585160e+00]],[[8.2857557e-02,-6.4077282e-01,-6.7505640e-01],[-2.3219872e-02,4.2155072e-02,-1.5368451e-01],[-7.9618198e-01,4.9755120e-01,-3.5868913e-01]],[[8.4251928e-01,-1.0085379e-01,8.7845400e-02],[5.5046600e-01,-6.4197883e-02,1.7548436e-01],[-4.2618859e-01,5.3132015e-01,4.1099411e-01]],[[-2.1775927e-01,-1.8216185e-01,5.8659899e-01],[1.0797574e-01,-5.5659986e-01,6.7542320e-01],[-9.7269732e-01,5.4796928e-01,9.8688507e-01]],[[5.5632704e-01,5.3394120e-02,5.5661225e-01],[4.9742845e-01,1.0324740e-01,1.0476359e-01],[6.6520989e-01,3.4436658e-01,5.1433754e-01]],[[2.9103784e-02,6.7647111e-01,3.6766064e-01],[-1.6465272e-01,4.2738432e-01,2.4124788e-01],[6.7778701e-01,-4.2342684e-01,6.1646676e-01]],[[-3.7940237e-01,-1.6143890e-01,-6.4591438e-01],[-5.3497541e-01,-4.3310288e-01,-3.7207714e-01],[-5.8857602e-01,-2.5836676e-01,-5.4723728e-01]],[[5.2856719e-01,3.6484784e-01,3.3206007e-01],[5.4666805e-01,-6.2309623e-01,6.4867699e-01],[-2.1774015e-01,1.2916595e+00,-6.2835234e-01]],[[1.7878667e-01,7.4182354e-02,-4.8201644e-01],[3.1676501e-01,1.1765845e-01,-3.6765181e-03],[1.0682367e+00,4.3881276e-01,9.7044837e-01]]],[[[3.50487947e-01,-1.52375177e-01,3.96697968e-01],[-3.22780246e-03,4.81177688e-01,1.34008750e-02],[-2.61534721e-01,4.09465164e-01,4.84211117e-01]],[[3.84521306e-01,4.14426476e-01,-2.47024894e-01],[-1.29027888e-01,5.25088787e-01,-7.12591708e-01],[9.34711993e-01,1.66895762e-01,1.29297721e+00]],[[2.33668596e-01,1.88867077e-01,-1.94935396e-01],[-1.39321446e-01,-3.95431191e-01,-4.45784815e-02],[-4.91499960e-01,-5.52592516e-01,-6.07111566e-02]],[[-4.81345914e-02,-1.83605179e-01,6.01196945e-01],[-1.08561717e-01,-3.18017118e-02,-2.83790201e-01],[2.66356856e-01,-6.18224025e-01,6.56166017e-01]],[[6.77983016e-02,4.71152604e-01,-8.39672387e-01],[1.43092588e-01,3.12950671e-01,1.01826000e+00],[4.94057536e-01,1.59338701e+00,3.67681503e-01]],[[-2.73385555e-01,7.92799443e-02,-4.22398329e-01],[5.80430448e-01,4.32137072e-01,-8.00902396e-02],[-9.99699906e-03,-1.16099787e+00,-1.50616455e+00]],[[1.06371328e-01,1.12123974e-01,-3.10188178e-02],[1.53385371e-01,-2.14903876e-02,6.54432923e-02],[-2.02614665e-01,-3.51908058e-01,-6.74259186e-01]],[[2.66133267e-02,1.10570692e-01,7.96625495e-01],[-1.47092387e-01,-1.44275784e-01,-1.03811985e-02],[-7.04041839e-01,-5.28458767e-02,-4.97217923e-01]],[[4.11449134e-01,5.57663321e-01,1.97368145e-01],[2.15624109e-01,-1.58320670e-03,-4.36965615e-01],[6.45657182e-01,1.07228017e+00,-3.66171300e-01]],[[-5.90370595e-01,-2.15146929e-01,3.82258445e-01],[-3.77160281e-01,6.04810007e-02,-4.51008439e-01],[2.16293573e-01,-5.54684281e-01,-5.56434929e-01]],[[-8.60011280e-02,6.39645709e-03,-8.09083045e-01],[-4.04435784e-01,-3.06865573e-01,-4.25997488e-02],[-2.06496328e-01,-2.22545713e-01,-9.72046316e-01]],[[4.78516787e-01,1.25647813e-01,3.56210351e-01],[-2.58610696e-01,-8.54079276e-02,-2.65475065e-01],[6.70884371e-01,7.65315235e-01,1.82994449e+00]],[[2.45534837e-01,5.93585987e-03,2.50191748e-01],[7.69408047e-02,5.92227340e-01,4.57974702e-01],[-6.12240672e-01,-2.83833668e-02,-5.96112311e-01]],[[1.35487124e-01,-1.10525936e-01,4.02386457e-01],[1.07932858e-01,-2.04241648e-02,-2.12976605e-01],[5.35829782e-01,-7.69389808e-01,-1.12995005e+00]],[[2.96429861e-02,2.56368428e-01,-1.29984859e-02],[-1.50239199e-01,1.99120998e-01,2.29396686e-01],[3.63746256e-01,8.16158056e-02,-8.38523209e-01]],[[3.25767040e-01,6.50519133e-02,5.07133067e-01],[4.92489636e-01,5.23528457e-01,8.25290322e-01],[-2.73004860e-01,-8.12397242e-01,-2.22564191e-01]],[[-1.90441370e-01,-3.33452880e-01,-4.34689105e-01],[2.99210221e-01,-6.28405288e-02,4.18223917e-01],[-1.41421810e-01,-3.29445541e-01,-2.22243741e-01]],[[-3.07198137e-01,-7.75769213e-03,-1.35145649e-01],[-2.60173053e-01,-2.48240739e-01,-1.72294617e-01],[3.23522031e-01,3.34344685e-01,1.39165032e+00]],[[1.02103725e-01,-9.95464325e-02,-1.08222947e-01],[4.67685759e-01,-1.03615813e-01,1.75850049e-01],[-1.08985724e-02,1.00959802e+00,-7.84109712e-01]],[[-4.38236952e-01,-4.65211064e-01,-1.89203069e-01],[-5.43204546e-01,3.11903864e-01,8.74682441e-02],[-5.00886235e-03,9.49111104e-01,-3.51997465e-01]]],[[[5.92729688e-01,-1.89761251e-01,3.39210331e-01],[-3.88476431e-01,1.17856175e-01,-1.46024644e-01],[2.52078593e-01,-2.21493579e-02,1.18752301e+00]],[[5.22074044e-01,7.83077031e-02,4.49982554e-01],[-3.81887674e-01,-4.50974941e-01,2.12151974e-01],[3.47316414e-01,-7.99041629e-01,5.87929487e-01]],[[-4.85280082e-02,-1.08181931e-01,1.38443828e-01],[2.77375787e-01,-6.61385283e-02,-2.74180651e-01],[8.24255347e-01,4.58463877e-01,1.83635294e+00]],[[4.41296659e-02,-2.82265455e-01,3.14629167e-01],[-2.32359152e-02,3.71657200e-02,-2.35979650e-02],[2.50932604e-01,-4.50223148e-01,1.67626357e+00]],[[-3.31366926e-01,-4.88086119e-02,-3.45367342e-01],[-1.91664129e-01,-3.93122397e-02,2.00367033e-01],[-2.76394337e-01,-1.43298006e+00,-6.25610113e-01]],[[-1.91343009e-01,2.64394701e-01,-5.50260782e-01],[-1.79352611e-01,-1.30839631e-01,2.34492674e-01],[-2.76000321e-01,3.95403206e-02,6.05317593e-01]],[[2.17781797e-01,4.28080499e-01,-8.79576802e-01],[3.09181005e-01,7.32938275e-02,2.27728650e-01],[-5.18090248e-01,3.25583458e-01,4.74966824e-01]],[[5.73727936e-02,1.09957077e-01,6.81750298e-01],[7.36774504e-02,5.73010266e-01,2.21652970e-01],[4.46677983e-01,-1.65134668e-02,2.59052485e-01]],[[-3.77528459e-01,1.28485933e-01,4.00656492e-01],[-1.02844782e-01,-4.19218689e-01,1.44760698e-01],[2.10874677e-01,-5.09181730e-02,-8.92220363e-02]],[[2.37117156e-01,1.64186195e-01,-7.48423338e-02],[1.93230752e-02,-2.45499790e-01,1.89215109e-01],[1.89140275e-01,1.21144257e-01,5.63703597e-01]],[[-5.83595186e-02,1.57556847e-01,-2.39887252e-01],[5.98889999e-02,-1.65803153e-02,1.61239877e-01],[1.84614316e-01,-5.87863252e-02,-1.17234957e+00]],[[1.33453635e-02,-5.62313721e-02,-1.53891966e-01],[-2.62555420e-01,-2.76596516e-01,-6.46092415e-01],[2.61931449e-01,-3.17685187e-01,1.62889779e+00]],[[2.45831683e-01,-2.01949731e-01,-3.00613493e-01],[-4.36444551e-01,2.42396340e-01,7.90287778e-02],[5.12729466e-01,4.00317669e-01,-2.48094842e-01]],[[-2.56773502e-01,-3.74884665e-01,3.51434052e-01],[4.35710698e-01,4.74763572e-01,1.21372424e-01],[1.13587730e-01,-1.37584105e-01,2.18730077e-01]],[[2.22795427e-01,-2.97938846e-02,1.13049895e-01],[-6.86014965e-02,9.07392874e-02,-2.38746777e-01],[7.08329260e-01,4.07828301e-01,-2.61401981e-01]],[[-4.30459976e-01,-6.31215692e-01,-4.92159128e-02],[-5.69407761e-01,2.24898800e-01,-2.85509229e-01],[-1.06906474e+00,-3.42889279e-02,-2.77525091e+00]],[[-1.40279084e-01,-2.25828737e-02,2.45503366e-01],[1.19125113e-01,1.48791850e-01,7.85762072e-02],[-1.31415951e+00,-9.26443338e-02,-1.14996541e+00]],[[6.55696571e-01,3.20675254e-01,2.53670197e-02],[7.29716718e-02,2.26202503e-01,2.04114720e-01],[8.59490454e-01,1.15598753e-01,1.35611463e+00]],[[-1.34804830e-01,-1.18494205e-01,-4.63613123e-01],[1.60088661e-04,-4.67673868e-01,-5.04249513e-01],[4.80911314e-01,-7.16232479e-01,4.91101056e-01]],[[4.07500677e-02,-1.33912206e-01,5.84342182e-02],[-4.52103794e-01,8.17722529e-02,-3.12405199e-01],[-4.84042764e-01,-4.87762809e-01,-1.94730163e+00]]],[[[0.17388763,0.07833447,-0.3582924],[-0.16725112,0.56228745,0.07563139],[0.23818886,0.05829541,0.28629854]],[[-0.2987,0.17128305,-1.5420661],[-0.3646422,-0.35584357,0.05882291],[0.29051575,0.2734213,0.3024213]],[[-0.74996334,-0.30493546,-0.00469784],[0.01442351,-0.21469015,-0.3253202],[-0.05931347,0.3666339,-0.27804127]],[[0.07414725,0.48661304,-1.5017965],[-0.00529096,-0.085806,0.5992658],[0.34420568,-0.27000245,-0.2962977]],[[-0.6323607,0.12672193,-0.34671822],[0.30240366,-0.01341336,0.17042314],[-0.46259075,0.65915716,-0.1932514]],[[0.70022994,-0.01149814,-0.2936422],[0.54682535,-0.25306204,0.06074516],[1.1017066,0.36906433,-0.53701633]],[[0.50169206,0.46407792,2.7008765],[-0.2337182,0.23385194,0.2949246],[0.1262929,-0.14247926,0.54165703]],[[0.29115197,0.08038967,-2.0142837],[-0.06701672,0.22686118,0.41647816],[0.166204,0.06302117,-0.7685867]],[[-0.42435354,-0.51112324,-1.1499224],[-0.11566706,0.4150679,-0.7104404],[-0.3096349,0.18532275,-0.10400829]],[[-0.64660823,0.36362445,-0.34015206],[-0.1583331,0.16006033,-0.39371508],[0.91601926,-0.12992415,0.01181509]],[[-0.13107228,0.1363347,2.64202],[-0.23690197,-0.04624717,0.05175833],[0.5863841,0.13638036,0.47917923]],[[-0.42082682,0.08054488,0.25003603],[0.17455354,0.36702096,-0.18281057],[-0.2878511,0.15314446,-0.23035575]],[[-0.11862434,0.2473971,0.11824142],[-0.05953932,-0.6699679,0.46347094],[0.29836342,0.34177953,-0.2622603]],[[0.27860326,0.20019014,-0.57751334],[-0.2251021,0.00751456,0.04864333],[-0.3751002,0.01852802,0.26557872]],[[0.39576098,-0.5306069,1.2339885],[-0.36752945,0.46714765,-0.341269],[0.71299714,0.11537484,-0.03714372]],[[-0.03049307,0.1979892,-0.50018054],[0.08590824,-0.00449236,0.23506652],[-0.00504533,0.13106085,0.52920884]],[[0.1516634,-0.35910416,0.09372678],[0.12740707,-0.07657911,0.07704021],[0.2986475,0.3695964,0.03137831]],[[-0.29785004,-0.06056852,0.14653212],[-0.19392227,0.23202345,-0.28173745],[-1.3492134,0.10371485,-0.7546038]],[[0.12098591,0.3712794,0.70006675],[0.26824462,-1.0368015,0.40079448],[-0.67529464,0.6753108,-0.412987]],[[0.2981027,0.13865505,-0.5411224],[0.14504822,0.10974032,-0.02262044],[-0.25439754,0.29528326,0.22516978]]],[[[1.0527971,-0.63587135,0.03710351],[-0.27787954,-0.17626931,-0.00606394],[0.5868186,-0.00592282,0.47361213]],[[0.40065965,-0.17064232,0.18995127],[-0.28669143,0.2086611,0.12918586],[0.13107647,-0.2083109,0.12244359]],[[0.37579608,-0.10595497,-0.21485795],[1.1582564,0.39585012,-0.00644143],[0.6248556,-0.6452222,0.22048008]],[[-0.19318081,0.0377287,0.00714619],[-0.28455704,0.16875917,0.00876937],[0.18401414,0.5572687,-0.17528735]],[[0.06121922,0.07158962,0.18385832],[-0.30076838,0.35990217,0.13715525],[-0.07803486,-0.69784796,-0.29523644]],[[-1.6443406,0.44584817,0.09908327],[0.2492842,0.01982433,-0.01641051],[0.28783822,0.05324609,-0.26488715]],[[-0.4828023,0.3772699,-0.13934712],[-0.20558389,-0.43080777,-0.11790281],[0.23343107,0.00453993,-0.07395267]],[[-1.6714731,-0.16556661,0.02243564],[-1.027956,-0.27585962,0.28999355],[-0.6082729,-0.03590437,0.1473286]],[[0.7050221,-0.26407617,0.23568898],[-0.7558961,0.11662751,-0.05254269],[-0.02210084,-0.14446177,-0.16800763]],[[1.826061,0.37336153,-0.2007525],[0.28798598,-0.04519526,-0.15974437],[0.44342992,-0.36416405,-0.2159521]],[[-0.6127931,0.19867562,-0.17624994],[1.2499158,0.3431382,0.22397363],[0.11916954,-0.35984886,-0.405453]],[[-0.64283556,-0.08115894,0.09429224],[-0.24885927,-0.14298446,-0.18291764],[-0.23009153,-0.00626937,0.27434704]],[[1.2979358,-0.35055602,0.2891944],[-0.51203454,-0.23457406,0.15521279],[0.22109206,-0.10074663,-0.3281974]],[[-1.6058153,0.3208181,0.30368513],[-0.25811207,-0.14258495,-0.03032193],[0.21340883,-0.8773095,-0.36153954]],[[-1.468858,-0.42903265,-0.12907596],[-0.154934,-0.369133,0.20694485],[0.47995675,0.14276432,-0.12689957]],[[-0.7420141,-0.3550162,0.0225487],[-0.7302091,0.18563771,-0.1943168],[0.11839653,-0.19763185,0.12847091]],[[0.16784354,0.31593344,0.02198139],[0.1311743,-0.27145612,0.1657487],[0.53307563,0.5102443,-0.33802742]],[[2.774261,-0.27264628,0.10039718],[-0.21761835,-0.4950821,0.00411052],[0.1709974,0.0747706,-0.08825368]],[[-0.20870948,0.3671011,-0.1780935],[0.37160468,0.26643002,0.46531695],[0.01356683,-0.6183027,-0.16325973]],[[-2.1570175,0.11739336,-0.16070466],[0.15427957,0.6269775,0.10814381],[-0.17314358,-0.4553873,0.242847]]],[[[0.44650033,0.23450014,1.4938998],[-0.17243496,-0.16088895,0.1538853],[-0.08290837,0.05420377,0.7814217]],[[0.21042599,-0.32126036,2.0483074],[-0.2218244,-0.4235037,0.15269914],[0.90955406,-0.23406275,0.6111578]],[[0.01806968,-0.05742355,-0.14061181],[0.23750544,0.08216526,-0.0124848],[-0.04873138,0.2745566,0.502396]],[[-0.01920492,-0.05404386,3.7786849],[-0.15781304,0.03039916,-0.5898126],[0.40386343,-0.33856344,0.44380346]],[[-0.49999747,-0.46778542,-0.6155264],[-0.19328971,0.3263123,-0.38975063],[0.09888006,-0.29618275,-0.34492493]],[[-0.02806984,0.02449565,-0.68981165],[0.14684528,-0.12411319,0.13181594],[0.73139447,0.15548967,-0.344223]],[[-0.17219758,0.49255514,0.15860146],[-0.0617377,0.06447559,-0.06063408],[0.3367172,0.20863448,-0.5323336]],[[0.38363457,-0.16387531,0.7873779],[-0.03462518,-0.0071328,-0.31841344],[0.3174612,-0.5024802,-0.586638]],[[-0.06799918,-0.14218614,-0.20419708],[-0.07099573,-0.31649494,0.0210724],[-0.56316596,0.46503773,0.38886386]],[[-0.06071144,-0.05893951,0.57121575],[-0.12364712,-0.33154792,0.19322233],[0.78928983,-0.30400935,0.3818639]],[[-0.5632968,0.0119204,-1.6258668],[0.00435857,0.2742892,-0.21327217],[0.2921475,0.11423121,-0.6003662]],[[0.15941152,0.43017647,0.14984192],[0.49615014,-0.03628209,0.05077084],[-0.6290925,0.10968413,0.22780083]],[[0.53448373,-0.22523877,0.8234949],[-0.06390065,0.22442108,-0.6721627],[-0.27501327,-0.06188787,-0.33003134]],[[0.0801213,-0.33740422,1.0331147],[-0.23429796,-0.10321177,-0.48348862],[0.23905325,0.11421924,0.19659096]],[[0.01829533,-0.14106654,-1.600531],[0.09440857,0.22253254,-0.33951652],[0.3847319,0.49274385,-0.5195148]],[[0.12053216,-0.21216711,-0.2534087],[-0.14105213,-0.08338366,-0.6104679],[-0.20282394,0.05389566,-0.90704405]],[[-0.31277552,-0.28221682,-0.31049207],[-0.12582229,0.04164697,0.08000262],[0.30395567,0.1655962,-0.47606018]],[[0.59593385,0.40452287,0.7357354],[0.00781279,-0.24261849,0.25406155],[-0.51762223,0.04816189,0.17528346]],[[0.3963385,-0.03118256,0.88236654],[-0.15447769,0.03768137,0.06648562],[-0.2302373,0.28349397,0.2865338]],[[-0.5527901,-0.48187977,-0.8383644],[-0.15940335,0.22311048,-0.12990065],[0.05519418,-0.26195234,-0.46747935]]],[[[-0.80238134,0.26104754,-0.354231],[-0.03145464,0.4586911,0.01547233],[-0.2568624,-0.00396477,-0.17236917]],[[0.29414546,0.50660557,0.66223735],[-0.3684012,-0.10746386,0.42180252],[-0.30008608,-0.36044884,0.10753511]],[[-0.18001641,-0.50421613,0.66936463],[0.35398558,0.2790558,0.04507592],[-0.30126777,0.4046737,0.6306409]],[[0.28712592,-0.44355625,-0.30372036],[0.21019845,-0.2762821,0.55860484],[0.19433819,0.09812491,0.57806236]],[[0.29639444,-0.1309969,-0.4066777],[-0.45968762,-0.58739036,0.64336115],[-0.6730819,0.1774882,-0.34840277]],[[0.92469484,-0.0426152,0.9297762],[0.0121963,0.30312067,0.5229754],[1.8711404,-0.18381585,0.65264726]],[[-0.2636606,-0.43824437,-0.8348454],[-0.7504416,0.42633867,0.3695425],[1.9556834,-0.25147408,0.5238084]],[[-0.33147714,0.13020948,-0.8262005],[0.10967815,-0.18425778,0.03231937],[0.43649063,0.46727338,0.852732]],[[-0.13027282,0.81809485,0.59584045],[0.21712889,-0.02915276,0.3165466],[-0.6560491,-0.53452694,-0.7580627]],[[-0.13132058,-0.71534836,-0.10363749],[-0.45279762,-0.00482553,-0.34434786],[1.0459177,-0.32143956,0.10899317]],[[-0.13227256,0.31598112,-0.08768775],[-0.06053881,0.04900946,-0.13805977],[0.06204728,0.45159093,-0.70678955]],[[0.10327739,-0.6865014,-0.4145011],[0.12923856,-0.11932275,-0.03831509],[-1.0975525,-0.01936303,0.16838497]],[[-0.55455387,0.41498807,0.02306432],[0.3498986,-0.34824914,0.13975811],[-1.1984092,0.32410717,-0.5945492]],[[0.6745058,-0.26692382,-0.11717739],[-0.32322976,0.01684207,0.49113712],[-0.05433161,0.83197266,0.33817825]],[[0.11630134,0.22554176,-0.06860794],[0.40331107,0.04582056,-0.19548596],[-0.671184,-0.06440005,-0.160698]],[[-0.0777662,-0.5261522,-0.07334495],[0.7220787,0.34875762,-0.64897704],[-0.01567226,-0.8197716,-1.0469558]],[[-0.02078216,-0.15769415,0.7405331],[0.12607342,0.39012134,-0.2480551],[0.454034,0.16552237,-0.26178738]],[[-0.50093096,0.09209455,-0.13876136],[-0.17351665,-0.2081516,-0.36761373],[-1.4681756,0.1460373,0.6353067]],[[0.02765606,-0.30398762,-0.20287873],[0.10060321,0.25069433,1.3122354],[-0.2164317,-0.11500625,-1.3437202]],[[0.37674132,-0.03852582,-0.3305084],[0.22181518,-0.18960512,-0.10870644],[0.10338449,0.2569082,-0.9431787]]],[[[1.02151155e+00,3.86077642e-01,5.21810770e-01],[1.11018419e-01,-2.84105569e-01,3.45950276e-01],[4.51278180e-01,-5.93372807e-02,7.25539386e-01]],[[2.36138806e-01,6.36824667e-01,1.12755013e+00],[3.46605301e-01,-1.92665160e-01,3.37767631e-01],[-8.21433604e-01,1.62859000e-02,8.97812247e-01]],[[7.86736682e-02,-1.00271732e-01,1.89466357e-01],[1.55911501e-02,3.99367511e-03,1.36021703e-01],[7.95389235e-01,3.17902893e-01,7.75303364e-01]],[[2.69690275e-01,2.52568930e-01,5.32265544e-01],[-9.40390304e-02,1.33021697e-01,1.27405420e-01],[-3.28528211e-02,-2.78674215e-01,9.46160078e-01]],[[-2.54016459e-01,1.83516353e-01,-7.23598339e-03],[1.35220498e-01,-1.96645036e-01,-5.50964236e-01],[-2.62881350e-02,6.57711387e-01,4.60718095e-01]],[[-7.21256852e-01,3.27262819e-01,1.45811409e-01],[8.92066658e-02,-1.11509480e-01,9.49483365e-07],[-8.19377482e-01,-7.99387991e-02,-8.90250802e-02]],[[3.80408528e-05,-1.00541256e-01,-4.80776548e-01],[1.96810156e-01,-2.69377977e-02,1.88893303e-01],[-3.44835639e-01,1.91634968e-01,-1.30137756e-01]],[[5.09086668e-01,-9.43860635e-02,3.31113189e-01],[4.51609224e-01,8.07441995e-02,-2.02374727e-01],[7.24895120e-01,1.31685242e-01,2.20712066e-01]],[[5.47725856e-01,8.63872841e-02,1.80142492e-01],[-1.27031514e-03,2.04390556e-01,1.57569051e-01],[7.23527968e-01,-1.74519464e-01,2.57617593e-01]],[[-3.92157167e-01,9.32926610e-02,-9.88752991e-02],[2.01485783e-01,2.31488459e-02,2.07621709e-01],[-1.18802762e+00,3.72612536e-01,1.67550191e-01]],[[-1.15252984e+00,-6.35479152e-01,-1.31496918e+00],[1.39309987e-01,1.58160046e-01,-6.07093126e-02],[-3.05755734e-01,-2.11312309e-01,-7.89063215e-01]],[[4.95064020e-01,-1.76219657e-01,-1.64824158e-01],[2.25375965e-01,-2.23718956e-01,-6.34070337e-02],[1.14001155e+00,-3.45965087e-01,7.06778646e-01]],[[1.05960667e+00,-2.65976079e-02,-2.80887008e-01],[3.08903098e-01,6.18000664e-02,-7.64785754e-03],[6.39849007e-01,-3.09750527e-01,3.75909299e-01]],[[1.93866715e-01,-5.52942939e-02,1.48073971e-01],[-4.71007586e-01,-4.17554557e-01,1.33252695e-01],[2.49914266e-02,5.98991752e-01,-5.19990802e-01]],[[-2.08853975e-01,-2.91265100e-01,-8.43392551e-01],[-2.10722342e-01,2.05819413e-01,1.86941892e-01],[2.64091015e-01,1.57669455e-01,-1.09699583e+00]],[[-2.60266572e-01,1.64256826e-01,8.07777345e-02],[-1.11620918e-01,8.94429907e-02,-4.17342007e-01],[3.19772899e-01,-2.47326598e-01,-9.55351472e-01]],[[-3.40919763e-01,-9.12771747e-02,-2.19525144e-01],[1.30197704e-01,7.71901533e-02,-1.77022353e-01],[-8.76737714e-01,1.50557488e-01,-9.49215889e-01]],[[1.24812543e+00,1.80563092e-01,6.78258359e-01],[4.29387212e-01,-2.39194557e-01,-7.70715773e-02],[1.49569261e+00,5.18669426e-01,9.05543625e-01]],[[4.09609005e-02,3.33113857e-02,-2.22502500e-01],[-3.34519118e-01,-5.10831654e-01,4.30602103e-01],[-5.84456384e-01,2.66163796e-01,7.73869097e-01]],[[-1.84102345e+00,-3.85962397e-01,-3.52049142e-01],[7.96633810e-02,-1.89313665e-01,-2.48100579e-01],[-6.34746611e-01,1.12505965e-01,-1.26513004e+00]]],[[[5.18434495e-02,4.33824271e-01,-1.46408290e-01],[1.93758279e-01,6.60687238e-02,-1.58468739e-03],[3.23580265e-01,-1.03796935e+00,-8.55714083e-02]],[[4.66712326e-01,9.68451574e-02,2.09006011e-01],[1.16478954e-03,1.45427048e-01,1.99651018e-01],[7.14675546e-01,9.40777063e-01,-9.90305245e-01]],[[-4.56279337e-01,3.69511425e-01,-9.27072823e-01],[4.29567605e-01,-2.49032572e-01,5.29038489e-01],[-7.01266378e-02,-1.06569850e+00,-1.11809336e-01]],[[1.40369624e-01,-3.05275410e-01,-5.61951697e-01],[-2.75510103e-01,4.89288062e-01,4.08723056e-02],[-2.35743493e-01,3.30311991e-02,4.66707855e-01]],[[4.74426240e-01,-2.42176950e-02,6.10976875e-01],[3.45545650e-01,-5.68761490e-02,-6.00414515e-01],[9.65207398e-01,9.19550478e-01,-6.28960013e-01]],[[-1.02944955e-01,5.47349416e-02,-2.64661342e-01],[-3.86197090e-01,-2.18284220e-01,2.69613832e-01],[1.18814312e-01,-8.69459867e-01,-7.23956347e-01]],[[1.54333666e-01,-2.09067151e-01,1.89675182e-01],[-9.49390326e-03,-4.33730155e-01,3.91626060e-01],[7.17097461e-01,-4.05363649e-01,3.69454086e-01]],[[4.41090673e-01,5.56363523e-01,-6.48362637e-02],[3.28095883e-01,-5.21575212e-01,2.50831157e-01],[-7.95283914e-01,-1.16172206e+00,8.14849555e-01]],[[1.79284111e-01,3.91923599e-02,-3.24842185e-01],[1.91009685e-01,-1.92248315e-01,3.02486807e-01],[3.86587024e-01,7.72896260e-02,-8.62225652e-01]],[[2.59387165e-01,3.79179180e-01,-2.41112247e-01],[-4.39124823e-01,-6.36423707e-01,-1.98579967e-01],[-4.77176756e-01,9.76413190e-01,-7.86247075e-01]],[[1.31028786e-01,-4.93543267e-01,3.08397442e-01],[-3.37399393e-01,-3.53005864e-02,-1.02241226e-01],[-6.71887472e-02,-5.12139976e-01,1.03236832e-01]],[[-2.47008696e-01,-3.09589297e-01,-1.77306980e-01],[2.87940800e-01,-3.23194474e-01,5.48174024e-01],[-4.10996556e-01,-7.74696589e-01,-4.05003339e-01]],[[3.32090706e-01,-5.41798353e-01,3.84760767e-01],[3.16404700e-01,6.64516211e-01,3.03995222e-01],[-5.99309325e-01,-8.69703710e-01,-6.31506145e-01]],[[1.06784567e-01,3.92800510e-01,-1.90567132e-02],[3.49655390e-01,1.85781881e-01,1.14208125e-01],[4.02838051e-01,6.45970225e-01,2.04689845e-01]],[[1.11065291e-01,-1.67732388e-01,4.25453871e-01],[-3.97093058e-01,-3.56428295e-01,-7.14020371e-01],[-5.27262509e-01,2.70430893e-01,-8.71841684e-02]],[[-6.84783682e-02,5.15072525e-01,6.53678970e-03],[-2.17127532e-01,-3.39472406e-02,-4.78237271e-02],[2.68998835e-02,1.00071681e+00,1.71933949e+00]],[[-6.87227994e-02,-3.01411420e-01,-2.80862927e-01],[-9.80382979e-01,3.30563962e-01,3.10865879e-01],[1.12527788e+00,-3.69632840e-01,1.01815438e+00]],[[-8.26713145e-01,1.19196452e-01,-4.67295885e-01],[2.78164566e-01,3.43574971e-01,-2.92849094e-02],[-5.59688985e-01,-4.98150170e-01,-5.16436756e-01]],[[-9.47631225e-02,-1.08950071e-01,-9.41720963e-01],[3.71787727e-01,-2.39327207e-01,-1.21650562e-01],[6.22755587e-01,8.74769986e-01,3.69312853e-01]],[[6.09862506e-01,6.52693927e-01,5.11913359e-01],[3.29495788e-01,3.84103000e-01,-1.49923220e-01],[3.19677234e-01,1.07386053e+00,4.97506887e-01]]],[[[-8.61056268e-01,1.08598247e-01,2.33277127e-01],[-3.99628311e-01,-7.07900226e-01,3.09967130e-01],[6.25571787e-01,-2.27392614e-01,-1.25935063e-01]],[[-3.17671537e-01,-4.71695364e-01,1.03886223e+00],[2.64822960e-01,-1.10187300e-01,-2.89847590e-02],[-3.77732038e-01,1.18463203e-01,1.34288871e+00]],[[5.06962985e-02,2.64085591e-01,3.77462618e-02],[-2.40356162e-01,4.84029762e-02,-3.69068772e-01],[-1.22628547e-01,2.31429920e-01,1.13389528e+00]],[[3.58099014e-01,-1.65183797e-01,1.44480002e+00],[-4.11675759e-02,1.94956386e-03,4.03872341e-01],[-3.54547292e-01,-4.62440908e-01,1.68018746e+00]],[[3.79142046e-01,1.87892094e-01,-7.58176625e-01],[3.67585063e-01,-1.51243642e-01,5.93016028e-01],[4.10908461e-01,-2.11813413e-02,-4.32961911e-01]],[[8.32454190e-02,-4.86216813e-01,-3.38877082e-01],[-6.22442126e-01,-8.61520618e-02,2.74570048e-01],[-5.46580911e-01,-5.35193563e-01,8.01788047e-02]],[[2.28429824e-01,9.20533687e-02,-5.01579285e-01],[1.57849789e-01,6.35783255e-01,4.67833787e-01],[-9.20044556e-02,-2.66111165e-01,5.57072135e-03]],[[-4.46857125e-01,-1.55184850e-01,2.39983518e-02],[-1.13437772e-01,-6.87477142e-02,-1.56987067e-02],[-8.48649561e-01,-4.44844514e-01,-8.92690420e-02]],[[4.42059249e-01,-4.22882773e-02,9.54274163e-02],[1.13338284e-01,6.62880167e-02,5.99801242e-01],[-8.03936347e-02,1.37600470e-02,-7.37691581e-01]],[[5.42448342e-01,-2.40500346e-01,9.58172023e-01],[-2.23030731e-01,-3.49959105e-01,5.25117576e-01],[-5.16133845e-01,-6.55281916e-02,2.52187371e-01]],[[6.26319289e-01,5.37037812e-02,-1.44806325e+00],[5.67468107e-01,2.28097260e-01,1.75418347e-01],[-4.15369928e-01,3.93154621e-02,-4.58154947e-01]],[[3.75875682e-01,4.45346981e-01,-4.19239610e-01],[-2.89344788e-02,-3.09100300e-01,-4.94683906e-02],[5.34531772e-01,-2.77987510e-01,1.04099917e+00]],[[1.11806743e-01,-2.41988394e-02,1.12496950e-01],[-4.58170325e-01,-3.21241558e-01,4.38615739e-01],[-8.39429572e-02,4.98725809e-02,-1.04930818e+00]],[[-1.13544011e+00,3.64332885e-01,1.03571050e-01],[-4.98959810e-01,-4.50541794e-01,-1.14063688e-01],[-3.03210914e-01,8.30912665e-02,-1.67393289e-03]],[[-3.47161293e-01,9.72726271e-02,-1.48473847e+00],[-4.12898660e-01,9.79134813e-02,5.71011286e-03],[-7.17005193e-01,1.52216673e-01,-3.17960978e-01]],[[-4.72477853e-01,4.29886132e-01,-2.34510601e-01],[4.22054797e-01,5.13887823e-01,-3.23294193e-01],[9.34158489e-02,6.26831710e-01,-8.95964503e-01]],[[-2.57770196e-02,3.30066472e-01,-2.74522930e-01],[9.49407965e-02,2.76871473e-01,1.24483846e-01],[3.78085792e-01,-5.88198960e-01,-6.62185550e-01]],[[-4.23052728e-01,6.10088110e-01,2.13925153e-01],[2.24779564e-04,-1.44793540e-01,-2.58100331e-01],[2.40687847e-01,-6.74104244e-02,1.45117378e+00]],[[3.33687037e-01,-3.47951323e-01,2.85700858e-01],[5.28942168e-01,3.58684063e-01,-4.36288193e-02],[7.04469979e-01,-7.51537308e-02,-1.11775585e-02]],[[-3.83800119e-02,-1.84614465e-01,-7.79384911e-01],[3.46109629e-01,-1.92631796e-01,2.11694390e-01],[-4.49324518e-01,2.10746065e-01,-1.49146283e+00]]],[[[2.31703296e-01,6.12155974e-01,-2.19407588e-01],[6.46109104e-01,5.34328640e-01,1.07566261e+00],[8.22922051e-01,5.55835426e-01,8.95408213e-01]],[[3.27006340e-01,-1.06935129e-01,3.24972630e-01],[-3.23236249e-02,2.53128171e-01,4.51072812e-01],[2.44918987e-01,2.52077401e-01,6.18376136e-01]],[[1.72185570e-01,-2.16918081e-01,3.82511437e-01],[5.92261732e-01,4.00368780e-01,-3.32623050e-02],[9.08035994e-01,6.38327654e-03,1.47383285e+00]],[[2.41463594e-02,-4.75770652e-01,-4.85942572e-01],[-3.87996763e-01,1.92951456e-01,-6.31187499e-01],[1.23116588e-02,-6.50518090e-02,3.45067739e-01]],[[8.77390876e-02,2.89692253e-01,5.01940072e-01],[1.82611302e-01,2.37516418e-01,5.69061220e-01],[8.79093230e-01,4.11027312e-01,-1.01199007e+00]],[[-3.51468176e-01,1.28152415e-01,2.93616336e-02],[2.59611309e-01,-5.74417114e-01,9.08473507e-04],[-9.27638352e-01,-5.39446890e-01,-6.68913603e-01]],[[2.01662838e-01,-2.38293156e-01,-1.70725852e-01],[1.01395294e-01,1.90707654e-01,2.39251763e-01],[-1.25291079e-01,-4.44860548e-01,-1.67210922e-01]],[[2.32513137e-02,1.82949603e-01,4.01909739e-01],[3.95260472e-03,-3.05401266e-01,-7.09634900e-01],[-7.76624203e-01,-3.66310626e-01,-1.89304852e+00]],[[2.43442431e-01,-3.80065918e-01,5.66297770e-01],[-4.23875064e-01,4.56150509e-02,1.42155319e-01],[3.10227662e-01,-2.14926362e-01,8.21118057e-01]],[[-1.34568345e-02,1.28221884e-01,-4.63174954e-02],[2.00203136e-01,-5.72211087e-01,3.20902735e-01],[-1.40757072e+00,-4.94024426e-01,-1.14358640e+00]],[[6.82567477e-01,-8.64222497e-02,-4.73135769e-01],[-6.03161082e-02,3.17965299e-01,-1.01138324e-01],[-1.32546321e-01,-9.49950814e-01,-1.44487226e+00]],[[-4.20033127e-01,4.89295274e-02,-7.37134576e-01],[-4.98964608e-01,-6.93922266e-02,-4.57275748e-01],[8.31300855e-01,1.55278906e-01,4.89130616e-01]],[[3.45718831e-01,2.23815560e-01,-2.30425850e-01],[-2.33590901e-01,-9.84893069e-02,1.66798085e-01],[-8.31738293e-01,-6.75361753e-01,-1.46370560e-01]],[[8.92161876e-02,5.56765758e-02,4.40748304e-01],[3.79883319e-01,-4.67372090e-02,6.56877398e-01],[3.71259570e-01,-7.79154837e-01,3.10286313e-01]],[[-2.82010883e-01,-6.27486482e-02,-3.32801014e-01],[-5.33333778e-01,-1.60551712e-01,-5.31223118e-01],[1.92012921e-01,-3.56435776e-01,-1.46647167e+00]],[[-2.48383731e-01,-4.76232201e-01,1.82255611e-01],[4.73604023e-01,3.54791284e-01,1.03585824e-01],[-1.92235395e-01,7.33935162e-02,-3.62367071e-02]],[[4.17978048e-01,2.51039058e-01,-3.27587575e-01],[5.06491773e-02,-1.79063505e-03,2.21116543e-01],[-4.57179360e-03,4.68904644e-01,-6.37542248e-01]],[[-8.36745426e-02,2.08371729e-01,4.07017730e-02],[3.04221839e-01,-4.16414589e-01,-5.79134643e-01],[1.73280731e-01,5.15772581e-01,6.04822874e-01]],[[-3.69935185e-02,-1.86534807e-01,1.64619535e-01],[-1.27798572e-01,-2.87483424e-01,3.11941117e-01],[2.86903441e-01,9.94024575e-02,6.95900321e-01]],[[3.08704704e-01,6.58164546e-02,1.71822459e-01],[3.79949987e-01,7.08384275e-01,8.57983232e-01],[-2.41224363e-01,-2.64162093e-01,-9.23504233e-01]]],[[[-6.5369177e-01,-1.2131871e-01,-5.3690207e-01],[6.2687621e-02,3.5708022e-01,-4.3980006e-02],[3.1514397e-01,-1.3352101e-01,-4.9774843e-01]],[[9.3291558e-02,8.0316849e-02,-1.5311773e+00],[-6.5040752e-02,3.1115609e-01,-4.1520000e-01],[-5.0437409e-01,6.3152421e-01,-8.1841028e-01]],[[-3.6970690e-02,-2.3053356e-01,-9.2722155e-02],[-4.0868682e-01,1.7828856e-01,-3.7978705e-02],[1.2753515e-02,-2.4664690e-01,-3.3394367e-01]],[[-1.5061156e-01,-2.7192584e-01,-3.6945734e+00],[1.0970090e-01,2.1926998e-01,2.7610269e-01],[-3.2490915e-01,5.7053453e-01,-4.3213052e-01]],[[3.5593975e-01,7.5220786e-02,-1.9464236e-01],[-5.1404301e-02,-2.3987280e-01,-1.4015457e-01],[-3.1935698e-01,6.7076527e-02,1.8344820e-01]],[[6.1649006e-02,-2.6323676e-01,7.1491772e-01],[2.7298044e-02,6.7980140e-02,-7.6024503e-02],[-9.7635025e-01,-1.7543359e-01,1.2434263e-01]],[[-2.2259504e-01,-3.8632724e-01,6.3468331e-01],[3.1839088e-02,-6.2044505e-02,6.3636190e-01],[-1.4380537e-01,-2.3506927e-01,3.4063151e-01]],[[-2.1498156e-01,1.4746068e-01,-6.6530496e-01],[3.3766398e-01,2.4764480e-02,7.2325683e-01],[-7.3316175e-01,3.5361671e-01,2.3654138e-01]],[[3.0530283e-01,2.6788294e-01,-3.0879155e-01],[1.0679659e-01,3.8927186e-01,6.1273780e-03],[5.5139941e-01,-5.8525246e-01,-2.6596701e-01]],[[1.6896990e-01,2.8960530e-02,-4.5318934e-01],[5.7292151e-01,2.2757350e-01,-3.0746841e-01],[-6.9322795e-01,-8.1633776e-02,-4.9015872e-02]],[[6.7024881e-01,2.0118597e-01,2.0312614e+00],[-5.3662863e-02,-2.0565385e-01,5.2773263e-02],[-5.5218953e-01,-2.3457937e-01,4.8098683e-01]],[[7.8791741e-04,-3.5606623e-01,-4.6467060e-01],[-3.3875763e-01,-7.7549353e-02,-6.0862564e-02],[2.8748998e-01,-2.4262384e-01,-9.2995799e-01]],[[-4.2124942e-01,2.4429597e-01,-1.0046291e+00],[1.9031772e-01,-3.9649469e-01,1.1854759e-01],[2.0393440e-01,-4.0471863e-02,2.5532493e-01]],[[1.9655718e-01,2.5832924e-01,-9.0236586e-01],[4.1402158e-01,5.0605081e-02,1.3947453e-01],[-4.2187080e-02,-4.8563963e-01,-1.5669012e-01]],[[-3.3267956e-02,-1.7997462e-02,2.0682173e+00],[-1.7444640e-01,-4.5302173e-01,8.0198848e-01],[-9.9175364e-02,-4.7346720e-01,5.8949518e-01]],[[-5.9239093e-02,-7.2419211e-02,-3.0806482e-01],[9.6817724e-02,1.5343429e-01,3.2080790e-01],[1.9725718e-01,-4.2293057e-01,6.6946787e-01]],[[5.0619036e-01,4.2953435e-01,1.5522879e+00],[1.5193573e-01,9.3989328e-02,1.7621371e-01],[-3.1045893e-01,-3.8921762e-01,6.2049299e-01]],[[-3.4869039e-01,-4.2448822e-01,-6.8742913e-01],[-2.0455655e-01,2.7018681e-03,7.1173068e-04],[7.6725483e-02,-1.9446641e-01,-1.8833528e-01]],[[-1.3176206e-01,-6.9315009e-02,5.4970670e-02],[1.2584855e-01,-8.0317393e-02,-2.9908201e-01],[-2.5515679e-01,-9.7630210e-02,-3.4122398e-01]],[[5.0970352e-01,3.7138274e-01,-2.6649585e-01],[3.4981981e-02,-4.4332527e-02,1.9077523e-01],[-4.1933545e-01,-7.4073076e-02,6.3453168e-01]]],[[[-0.97314095,-0.25948697,-0.08190658],[0.01276616,-0.21268743,-0.5611527],[-1.113313,-0.6446535,0.09484266]],[[-0.07123607,-0.42751667,0.2072216],[-0.28330347,0.08599872,-0.26242626],[-0.54897135,-0.10375485,-0.48524606]],[[-0.02929384,-0.1751034,-0.30708623],[-0.51472765,0.26756522,-0.3735835],[-0.3549115,-0.12358409,0.51612294]],[[0.09600733,0.09175591,0.23539723],[0.2812042,-0.16878112,-0.02574735],[0.2512297,0.38739896,-0.03882617]],[[0.00966329,-0.33224738,-0.59073323],[0.402676,0.17961156,-1.0050032],[0.12845449,-1.354043,-1.374031]],[[0.5978771,0.08744827,0.09188718],[0.0420589,0.00797023,0.31260514],[0.69877803,-0.50152206,0.21330683]],[[0.29447737,0.47652224,0.90819925],[-0.11179738,0.25577682,-0.33976978],[0.7509306,-0.6603848,0.87140244]],[[-0.10873739,0.04974397,0.4354916],[0.34132734,0.1710103,0.33684763],[-0.8944971,0.4011223,0.00349973]],[[-0.00428836,0.5792383,-1.2983388],[0.30512437,0.06465488,-0.10079626],[-0.5076666,-0.6904269,0.3104748]],[[-0.5367821,-0.5292841,0.39236897],[-0.09078744,0.28404003,-0.21392158],[0.36358646,0.4371376,-0.7446428]],[[0.33059236,0.3242711,0.13629344],[-0.2640995,-0.24502878,0.33201665],[-0.28635004,-0.0893018,0.63582253]],[[0.1699836,-0.3425816,1.1374948],[-0.32868883,-0.17127384,-0.0479875],[-0.7998529,-0.33602062,-0.42344818]],[[-1.415648,0.41191944,-0.6940277],[0.17354463,0.3101763,0.34397766],[-1.9318594,0.04824667,0.49739859]],[[0.02783339,0.00395035,0.36382836],[0.13386603,-0.14731765,0.09354577],[-0.427801,-0.51282465,-0.16607432]],[[0.25805888,0.43482035,-0.00814222],[-0.01285642,-0.40984863,-0.30042785],[-0.2568529,0.03837476,0.53677434]],[[0.5710276,0.19555958,0.13334969],[0.680794,-0.11967871,-0.2093843],[0.96023434,0.5399141,1.2275385]],[[-0.0919393,0.12272234,-0.00941293],[-0.0273612,-0.30575916,-0.06446192],[1.0166329,-0.3537852,0.68178153]],[[-1.1979539,-0.22764668,0.28701264],[-0.4015797,-0.02715072,0.02743442],[-0.9508218,-0.31638122,-0.67344016]],[[0.44080603,-0.28729403,0.6358979],[-0.5705598,0.51517427,-0.34074348],[0.5471977,-0.24740757,-0.04292047]],[[0.98024493,0.17395565,0.30563933],[0.6108677,-0.47284573,0.30014578],[1.8466113,0.3876868,0.9539896]]],[[[-1.90144822e-01,-2.74030954e-01,1.50457099e-01],[2.29568586e-01,-2.33497292e-01,-3.79896104e-01],[3.24652046e-02,-1.32647097e-01,-3.89930420e-02]],[[-1.04098225e+00,-9.62556660e-01,-4.59262490e-01],[-3.15317988e-01,7.46330852e-03,1.76487342e-01],[3.26961726e-01,5.64394712e-01,-2.24234626e-01]],[[-4.51975703e-01,1.23494720e+00,4.46348578e-01],[1.81195572e-01,4.00908977e-01,-1.87776715e-01],[2.38160402e-01,-1.53983212e+00,-1.44821241e-01]],[[4.27991480e-01,5.62410593e-01,1.22398287e-01],[6.44329071e-01,1.63805321e-01,-7.81867132e-02],[-2.81000793e-01,6.79509640e-01,-5.01316428e-01]],[[-5.08709729e-01,6.39778972e-01,5.17774582e-01],[-4.99563724e-01,6.24349304e-02,2.67206877e-01],[-2.25687586e-02,-2.03147650e-01,-1.01362026e+00]],[[6.95304871e-01,-4.34102327e-01,-2.95487374e-01],[1.34853022e-02,2.41880476e-01,-4.59106565e-02],[8.19250107e-01,-9.85145032e-01,3.16821516e-01]],[[2.00033784e+00,-7.72034824e-01,-1.11897841e-01],[-9.92596507e-01,2.25204483e-01,3.49636793e-01],[-2.93203980e-01,-3.53757769e-01,-5.73025644e-01]],[[3.43164533e-01,-1.00263727e+00,7.23498166e-01],[1.98182464e-01,-4.80319053e-01,-1.65769413e-01],[-3.74641746e-01,-2.09304476e+00,4.52330023e-01]],[[-6.83829784e-01,-5.12070954e-01,3.04626897e-02],[-6.57174885e-01,-4.86401826e-01,2.49546051e-01],[-5.68935394e-01,-1.64204732e-01,-7.17453286e-02]],[[-4.82578009e-01,9.55543220e-01,3.65408182e-01],[-9.94348805e-03,-6.28962591e-02,-4.33356553e-01],[4.41987962e-01,2.87723243e-01,3.15155424e-02]],[[3.99070531e-01,1.23002279e+00,5.31025469e-01],[-9.68021806e-03,2.51097754e-02,3.48271914e-02],[2.34317720e-01,3.43621299e-02,7.03331649e-01]],[[1.11434448e+00,-7.14630663e-01,-9.81044546e-02],[2.91896850e-01,-1.08355932e-01,2.78323919e-01],[1.70014024e-01,-4.23819393e-01,-7.77714014e-01]],[[4.45339441e-01,4.10838723e-01,9.14941669e-01],[3.52054089e-01,9.00003389e-02,-5.91322362e-01],[5.81707597e-01,-1.29182920e-01,-2.52194367e-02]],[[2.11248770e-01,-1.21493757e+00,-1.12203665e-01],[2.89160758e-01,-7.99115181e-01,-3.23645175e-01],[-3.42189997e-01,-4.12025958e-01,2.43998110e-01]],[[6.11219108e-01,-1.55683529e+00,-5.92994690e-01],[-2.28603587e-01,-2.25959256e-01,-2.05614954e-01],[4.29716825e-01,-1.99536398e-01,2.74378330e-01]],[[6.56964779e-01,-2.81951427e-01,-2.94715375e-01],[6.49965286e-01,5.43465436e-01,5.35731196e-01],[-6.32085025e-01,-7.36665905e-01,-4.34615463e-01]],[[-5.21911561e-01,2.20923513e-01,2.86076188e-01],[-6.95602074e-02,-2.57970899e-01,2.43467674e-01],[3.20904881e-01,3.43463063e-01,-3.96489277e-02]],[[4.11480755e-01,-2.22019270e-01,4.89123166e-01],[-2.57174313e-01,2.11334452e-01,-1.66696776e-03],[-6.69551015e-01,5.25879145e-01,-1.97939038e-01]],[[1.56215370e-01,-3.09274226e-01,-1.17456224e-02],[6.36162981e-02,1.54653743e-01,3.28604043e-01],[1.16099465e+00,-3.52087349e-01,-5.40056527e-01]],[[3.98461044e-01,5.60761914e-02,-2.13215739e-01],[-1.38457537e-01,-8.41884315e-02,2.40493461e-01],[5.95073402e-01,4.22945358e-02,-1.27488866e-01]]],[[[-7.3837243e-02,-2.3072128e-01,5.2222413e-01],[7.6980215e-01,8.1142563e-01,2.4603280e-01],[-1.6189203e-01,1.1536635e+00,4.4066720e-03]],[[4.2459032e-01,1.1666770e+00,-2.1764870e-01],[2.0005247e-01,4.9240503e-01,-1.3186835e-01],[-7.1474046e-01,1.2915012e+00,6.4322013e-01]],[[1.8999793e-01,5.7186675e-01,1.4597763e-01],[-2.8613520e-01,-3.1787783e-01,-8.8308549e-01],[1.2479968e+00,3.3996806e-01,-6.7369223e-01]],[[-3.0485716e-01,6.5991330e-01,-4.5272186e-01],[-5.5286932e-01,6.3048303e-02,-8.6542541e-01],[-2.1753076e-01,-4.5296052e-03,2.2859278e-01]],[[-1.6274709e-01,4.6960646e-01,-5.6824130e-01],[3.1960952e-01,2.5938785e-01,-8.5789096e-01],[7.6679158e-01,-9.2867059e-01,-7.8446537e-01]],[[1.5686020e-01,1.9511355e-01,-1.5929246e-01],[-4.1648246e-02,3.7222110e-02,1.0285136e+00],[-8.6728162e-01,3.5025886e-01,-3.1041297e-01]],[[3.0874131e-02,-5.2042794e-01,-8.3997853e-02],[4.0293317e-02,-5.7013072e-02,2.1517511e-01],[-1.8598227e-01,-5.4960126e-01,-7.6483703e-01]],[[-3.6111498e-01,5.8622304e-02,7.9131484e-01],[2.2295879e-01,-1.7490816e-01,-1.8960221e-01],[6.9266659e-01,-1.2314225e+00,-9.9637248e-02]],[[-2.4214423e-01,4.2296074e-02,-4.6874106e-01],[-9.7015277e-02,-4.0578821e-01,-1.9916105e-01],[2.0720103e-01,1.5271009e+00,8.7494439e-01]],[[3.3362362e-01,1.2933214e-01,-8.8330632e-01],[-9.1788329e-02,-7.7760324e-02,-4.0342602e-01],[-8.7331319e-01,4.2946771e-01,-2.1114123e+00]],[[-4.7384100e-03,-7.1012401e-03,-1.5487996e-01],[1.4984937e-01,-1.8417917e-01,2.3335306e-01],[-3.9391869e-01,-1.0951140e-01,-5.6948215e-01]],[[-4.9006790e-01,-4.7392315e-01,2.5280622e-01],[-2.5295883e-01,5.8234677e-02,-8.8262784e-01],[3.2787612e-01,-1.4594840e-01,6.5824407e-01]],[[-3.8023019e-01,-5.8609301e-01,4.1956240e-01],[1.0129690e-01,-2.4240315e-01,-3.5109693e-01],[-4.5792148e-01,3.2526845e-01,2.2966990e-01]],[[3.6402017e-01,6.7550644e-02,7.6484287e-01],[2.1906924e-01,-2.0301713e-01,-4.4763947e-01],[6.7544258e-01,-1.6835766e-01,-1.2903814e-01]],[[5.2899671e-01,-6.1469454e-01,7.6700687e-01],[-7.6141991e-02,2.8120956e-01,-1.2891090e-01],[-2.2696084e-01,1.0439612e+00,-4.4318515e-01]],[[-1.0173480e-02,-4.0314779e-01,-1.5040059e-01],[-4.1272444e-01,-3.3845109e-01,3.8711706e-01],[6.9638795e-01,-4.2826772e-02,-2.5709760e-01]],[[-4.1583818e-01,-5.0708592e-02,-1.9924456e-01],[-2.5217584e-01,4.8213410e-01,1.2854533e-01],[1.4081837e-02,-1.0293982e+00,-4.7981119e-01]],[[2.3513336e-01,1.8958101e-01,-2.2562376e-01],[-1.5950222e-01,-1.4875394e-01,4.8554814e-01],[4.1362506e-01,-4.1474068e-01,5.3335506e-01]],[[2.2839987e-01,-3.0729663e-02,-3.8786259e-01],[-2.7720958e-01,5.9839815e-01,7.3047018e-01],[4.8207122e-01,1.2939605e+00,4.7060111e-01]],[[1.0026357e-01,-2.2188673e-02,-9.2079127e-01],[7.2505331e-01,-6.5828276e-01,5.8853012e-01],[1.0636634e-03,4.6276298e-01,2.5826031e-01]]],[[[0.5087034,-0.3764539,0.35692173],[0.01798747,-0.05736879,0.18706696],[0.4573715,-0.25027883,0.80655366]],[[0.16926923,-0.74550563,0.396287],[-0.20418203,0.2891796,-0.40841687],[0.2747984,0.46949747,-0.06279656]],[[0.48129895,-0.07657356,-0.1752405],[-0.06459985,-0.33710253,-0.27802035],[0.05352384,-0.09653889,0.82874256]],[[0.73527044,0.4520005,0.9613163],[-0.20579322,0.1801133,0.04492836],[-0.29109964,-0.2777938,0.68510073]],[[-1.1818095,0.17790155,0.26269653],[-0.2187183,-0.03984252,-0.08787274],[-0.03256781,0.46938577,0.19091283]],[[-0.3550898,-1.2773871,-0.9913829],[0.11886552,-0.42491135,-0.57986236],[-1.1169615,-0.02708574,-0.67163825]],[[0.6153077,-0.42893362,-0.03854343],[0.2262913,0.29014727,0.36169016],[-1.0334048,0.01854772,-0.31427497]],[[0.6672367,-0.6669835,0.17175871],[-0.83633643,-0.59057385,-0.6931961],[-0.27760047,0.3033416,-0.1625191]],[[-1.3297191,0.02944605,-0.28324765],[0.18031405,-0.05139189,0.3122962],[0.7858417,-0.26529452,0.03731371]],[[0.47988153,1.0165654,1.1354738],[0.22346267,-0.274432,0.5192523],[-0.43404567,0.01954165,0.15331009]],[[-0.60184205,0.08518016,0.36603218],[0.5016719,0.5743064,-0.01112552],[0.07075553,-0.1079265,-0.8555957]],[[0.74419016,0.1891157,-0.25749782],[0.12206523,0.08081795,-0.37439767],[-0.18435852,-0.32258373,0.6753639]],[[0.73817635,-0.11599097,-0.1324849],[0.0243201,-0.04184935,0.71702],[0.12468694,-0.4762629,-0.40578544]],[[0.49088898,-1.1720086,-1.3732342],[-1.1634734,0.4811448,-0.58465004],[-1.2635756,0.62932056,0.11488497]],[[0.16892515,-0.25279605,-0.3541964],[0.11928805,-0.04410962,0.0997792],[-0.2443272,0.00556115,-0.27661937]],[[0.11264821,0.11084478,0.13482553],[-0.777461,0.8252944,-0.42241925],[-0.5997294,-0.3379464,0.10869764]],[[-1.150173,-0.0492726,-0.08698041],[0.00276113,0.253898,-0.55497223],[-0.53844583,-0.36150673,-0.43578967]],[[0.69642615,0.5767892,0.50709254],[-0.11381688,0.26200002,-0.16137184],[0.14050497,-0.06807117,0.8992197]],[[-0.40138036,-0.05534321,-0.8883233],[0.1551422,0.18637748,0.1784144],[0.3597209,-0.03571259,0.80395067]],[[-0.936183,-0.87907225,0.8052707],[0.28145835,0.8496907,0.02571693],[-0.39858732,-0.40204623,-0.31815818]]],[[[7.99131453e-01,1.94723904e-01,-5.12057602e-01],[-3.97043735e-01,-2.86596864e-01,-1.60308957e-01],[-2.41841257e-01,-6.37472048e-02,-1.08971190e+00]],[[6.71101153e-01,4.98953313e-01,-2.55722582e-01],[-1.82853919e-02,3.61825645e-01,-6.24793410e-01],[5.82596004e-01,4.17749226e-01,-7.48302996e-01]],[[2.73482442e-01,1.00686140e-01,3.34936142e-01],[-1.40607655e-01,4.87455606e-01,6.19966909e-03],[-8.48340034e-01,-9.09931421e-01,-1.47685754e+00]],[[-1.91070884e-01,7.55331293e-02,1.76357597e-01],[-4.21303540e-01,2.53898710e-01,-1.26948908e-01],[-4.27147120e-01,8.40564132e-01,-2.28999421e-01]],[[-1.36642069e-01,9.79555398e-02,-1.42736435e-01],[-1.62208930e-01,5.87818444e-01,7.06990540e-01],[2.42324382e-01,9.78415683e-02,-1.03404915e+00]],[[1.08068131e-01,1.70995772e-01,1.12413108e-01],[-1.05745685e+00,7.28559271e-02,-1.84873298e-01],[-1.45221516e-01,4.78680693e-02,2.10508076e-03]],[[-4.59492028e-01,-7.21205324e-02,6.01230562e-01],[-1.44713864e-01,-9.06273723e-02,-9.79348272e-03],[-9.70868647e-01,-3.20153087e-01,4.53962803e-01]],[[4.46508378e-01,7.74200680e-03,-5.47511756e-01],[4.32057947e-01,4.55534577e-01,9.01986728e-04],[-9.92910564e-01,-4.99277353e-01,-4.25196558e-01]],[[-4.27801788e-01,-1.39589384e-01,-3.61861438e-01],[-4.96700495e-01,-5.55585682e-01,-1.30702078e-01],[-2.17839256e-01,-2.66299874e-01,9.24857974e-01]],[[-6.84013069e-01,9.14469287e-02,4.16580111e-01],[-4.40840513e-01,4.91501153e-01,-6.57399714e-01],[-1.18387651e+00,7.95776367e-01,-6.12106860e-01]],[[-8.25927913e-01,-4.61877398e-02,4.78552788e-01],[-1.17120743e-01,-1.31098256e-01,5.53957298e-02],[5.11800706e-01,-3.49094987e-01,1.25156605e+00]],[[6.07239366e-01,1.32918641e-01,-2.94489086e-01],[1.16851956e-01,4.63596672e-01,-2.13575900e-01],[-5.86102366e-01,-3.59682411e-01,-1.76246238e+00]],[[6.91487789e-01,-1.60518691e-01,-3.40130925e-01],[-8.13244581e-02,-5.04103720e-01,4.71865714e-01],[2.75186211e-01,-2.84196764e-01,7.76172221e-01]],[[-5.60548842e-01,-1.07013986e-01,-4.19663072e-01],[-1.73207462e-01,2.73826858e-03,2.93585688e-01],[-3.55620563e-01,4.40936953e-01,-1.71358213e-01]],[[2.31958523e-01,-4.90107149e-01,1.28435171e+00],[-4.47365135e-01,-6.35777861e-02,-8.69812742e-02],[1.66203961e-01,-1.72994718e-01,1.61455929e+00]],[[-7.07815439e-02,5.35539567e-01,-4.28987026e-01],[-1.52975425e-01,-1.23781696e-01,3.21989268e-01],[1.27513021e-01,-8.18501830e-01,3.12119991e-01]],[[-1.34400129e-01,1.37535155e-01,-1.65324524e-01],[-4.31805342e-01,-1.00075729e-01,7.10631609e-02],[-1.35222778e-01,-4.10597362e-02,5.34545600e-01]],[[9.55016613e-01,-3.07284683e-01,-3.99483889e-01],[1.27089858e-01,-2.45868415e-02,1.09552771e-01],[2.32631043e-02,-4.70563143e-01,-3.62459242e-01]],[[2.62742132e-01,-3.64974231e-01,4.19519573e-01],[2.63627499e-01,8.11703026e-01,-8.55739713e-02],[5.52806497e-01,-1.04264513e-01,3.30825418e-01]],[[9.15377587e-03,5.24023511e-02,-2.20894460e-02],[-4.89924371e-01,-2.58925587e-01,2.17814595e-01],[-8.33439603e-02,9.32575464e-02,6.50289237e-01]]],[[[-6.6655248e-01,-2.0879559e-01,-2.2216149e-01],[-1.6184118e-01,1.3449888e-01,1.9633174e-01],[-4.0032867e-01,-5.1063937e-01,-1.3587933e+00]],[[-5.1327008e-01,9.3871146e-02,-3.4005392e-01],[-1.3021999e-02,4.1266268e-01,-7.9279971e-01],[8.3025821e-02,-3.7540939e-01,-2.1552025e-01]],[[4.1487801e-01,2.8819558e-01,-6.0730952e-01],[-9.5399618e-02,1.2359303e-01,-2.4194901e-01],[-5.4663676e-01,6.5183356e-02,-3.3003950e-01]],[[-2.5982630e-01,-1.6407160e-01,-2.9164001e-01],[1.1564586e-01,1.8794836e-01,-2.7311396e-02],[-9.0549253e-02,1.4393961e-01,-6.0858762e-01]],[[-8.8690735e-02,-3.7458730e-01,3.6055180e-01],[-3.6441749e-01,-4.3642271e-01,-6.7242396e-01],[7.7920866e-01,-6.7166343e-02,3.5012478e-01]],[[2.0417716e-01,3.5934791e-01,6.9348472e-01],[-2.9067522e-01,-9.7929634e-02,1.3539581e-01],[4.8140854e-01,1.4288174e-01,-1.5409546e-01]],[[-4.2696196e-01,3.0817166e-02,-2.6925984e-01],[2.1142305e-01,-2.3847258e-01,-4.3293394e-02],[5.7315791e-01,1.9037237e-03,1.1381242e+00]],[[-9.2794162e-01,-4.5271114e-02,-1.6214151e-02],[2.0243184e-01,-8.3863461e-01,5.2093154e-01],[6.3684827e-01,4.0787986e-01,-4.2075723e-01]],[[-2.7662697e-01,-4.5160890e-01,1.7143769e-02],[-1.5250498e-01,1.8442906e-01,1.3519251e-01],[-8.3119661e-01,7.0336066e-02,-4.0931079e-01]],[[1.7661180e-01,-3.3113968e-01,3.8853753e-01],[1.7225547e-01,2.5463521e-01,-1.1511545e-01],[9.7560185e-01,1.2681650e-01,-1.3895167e+00]],[[8.7712610e-01,-1.8082863e-01,4.0904820e-01],[-1.4104234e-01,-2.8726885e-01,5.3306725e-02],[1.5414506e-01,2.1888360e-01,1.3423654e+00]],[[-6.2547922e-01,8.6881919e-03,-1.4630742e-01],[3.1428468e-01,-1.7039068e-01,-2.0813119e-02],[-3.7206855e-01,-3.6698323e-01,-1.0957947e+00]],[[-3.6296532e-01,-4.0820163e-01,3.3095306e-01],[2.2781765e-01,-8.4997487e-01,5.4387224e-01],[-4.1819578e-01,7.3716396e-01,-8.4500957e-01]],[[-3.0083221e-01,1.8267095e-01,-1.9158942e-01],[4.5571763e-02,-3.4742564e-01,2.0831877e-01],[5.5259429e-02,3.2154435e-01,2.0811045e-01]],[[-3.0344328e-02,2.6394179e-01,2.7613643e-01],[2.7010897e-01,1.5033861e-01,2.9436779e-01],[-5.4071540e-01,-3.3439834e-02,1.3755922e-01]],[[-2.4118228e-01,1.7233336e-01,-2.6035288e-01],[4.3004494e-02,-2.1287690e-01,-9.3696304e-02],[-1.0059731e+00,-4.3152656e-02,1.0555698e+00]],[[6.2874871e-01,1.5023884e-01,4.5772561e-01],[-1.4338951e-01,1.7995985e-01,-1.3915758e-01],[4.5901754e-01,-5.4809380e-01,2.7720518e+00]],[[-6.3666832e-01,1.3201847e-02,1.5970451e-01],[-1.5663220e-01,1.2053869e-01,2.5136170e-01],[-7.0448585e-02,-1.3526899e-01,-1.5286510e+00]],[[-1.2771039e-01,3.1280831e-01,5.3153622e-01],[2.7975500e-01,3.3578598e-01,4.5475030e-01],[-2.5302789e-01,-6.1253893e-01,4.6639282e-01]],[[9.1930389e-01,3.7627468e-01,5.3644770e-01],[-2.7347350e-01,-1.6746466e-01,6.0819018e-01],[5.5406392e-01,4.7624925e-01,9.6925065e-02]]],[[[0.7613082,-0.1333465,-0.30894166],[0.23533273,-0.6600226,0.12340216],[-0.6259181,1.172387,-0.2879222]],[[0.50378937,0.62910384,-0.3634332],[-0.38514742,0.15328911,0.11158808],[0.19717675,-0.05892488,0.6701877]],[[-0.1364633,-0.18667167,0.71151274],[0.24291466,-0.853826,-0.8732348],[-0.227282,0.32665163,-0.54788226]],[[0.03746416,0.69853675,-0.71650755],[-0.68914723,0.63421917,-0.49094677],[-0.15662362,-0.74323606,0.7810059]],[[0.11643438,0.09130888,0.20051958],[0.29717305,0.05391341,-0.62594336],[0.09354149,1.2826952,-0.3921769]],[[-0.46753052,-0.04208808,-0.04629984],[-0.8665384,-0.6424931,0.41404808],[-0.09154289,0.4662397,-0.7359316]],[[-0.16519098,0.05860769,-0.878615],[-0.5834477,0.52748346,-0.03076804],[-0.1105722,-0.18737738,-0.5659344]],[[-0.39063904,0.33619946,0.13401006],[-0.01358907,-0.7949564,-0.3724164],[0.23856124,-0.19015488,-0.61270773]],[[0.5287131,-0.46103096,0.5980177],[0.32751283,0.8930601,0.14715154],[0.0904237,1.0180455,-0.5500588]],[[0.17218502,0.56624246,-0.46564105],[0.323809,0.00602584,0.04935058],[0.36001608,0.47808087,-0.21409051]],[[0.30411443,0.651532,0.458009],[-0.17194735,0.1459323,0.34803507],[-0.20123562,-0.8031005,-0.0196975]],[[-0.02158967,0.12154141,-0.18433186],[0.10885385,-0.41208273,-0.32421753],[-0.65653807,-0.21945418,0.7076118]],[[0.4532886,-0.47077194,-0.7019891],[-0.29030117,0.58043134,0.6941047],[0.69239956,0.29564455,0.25111568]],[[-0.65610135,-0.11797801,-0.7591568],[0.05522333,0.5237971,0.12878838],[0.61751956,0.409443,-1.7737167]],[[-0.1874718,-0.22349241,0.3586425],[0.17656936,0.3436824,-0.48194098],[-0.5308611,0.19491845,-1.0448182]],[[0.089586,-0.11149684,0.24856398],[-0.38818848,0.21258585,0.3914172],[-0.72276413,0.06336797,0.67485017]],[[-0.66458285,0.42129472,1.0625114],[-0.1779154,-0.6199782,0.6592194],[-0.06791814,-0.6557226,-0.30303234]],[[1.1472913,0.37639004,0.18229876],[0.16826619,-0.7524797,0.6317205],[0.24245659,-0.57565737,0.78816724]],[[-0.56119215,0.05279386,0.4251034],[0.6805235,0.6066779,-0.06482895],[0.42542535,0.17335618,-0.00302601]],[[0.47038043,0.10330339,-0.18961437],[-0.3760897,-0.032989,0.5312541],[-0.24239609,0.1935569,0.4650039]]],[[[-1.0175263,-0.00561978,0.7388263],[0.81651366,-0.2984872,-0.28335842],[-0.08445036,0.801931,-0.02400289]],[[-0.3497881,0.30130777,-0.27862474],[-0.7529222,0.40340683,-0.01639264],[-0.46432137,-0.131911,0.23289652]],[[-0.57793975,-0.10714059,0.48744196],[0.22222482,-0.1691434,0.10461516],[-0.10207233,0.21077013,-0.93275267]],[[-0.26198694,0.11449218,-0.62416464],[-0.49573958,-0.01223818,-0.94150436],[0.31231606,-0.13300812,0.4350841]],[[-0.31185162,-0.0274144,0.33674243],[-0.16407633,0.612258,-0.1920888],[0.40464252,-0.01946989,0.8172745]],[[0.87181956,0.4278576,-0.01406118],[0.27912575,-0.8596645,0.45935717],[0.9454152,-0.6659307,-0.04585525]],[[0.23164113,-0.36064973,-0.01888569],[-0.21420625,-0.516977,-0.12393047],[-0.07587856,-0.2923545,0.1925159]],[[-0.16119881,-0.13256319,0.3161459],[-0.11924037,0.36823764,-0.70245844],[-0.32899636,-0.4969962,0.4112083]],[[-0.7143586,-0.0599883,-0.28842258],[0.3199036,-0.36405882,0.4139241],[0.23467882,0.39884958,0.8669731]],[[-0.38764146,-0.03926041,-0.8120294],[0.6332093,-0.48650226,0.25285792],[1.226722,0.31048113,-0.01997011]],[[0.30245733,-0.32380423,0.13271435],[-0.5509404,-0.08267569,-0.840389],[-0.9371148,-0.70737284,0.44908285]],[[-0.6190429,-0.11360478,-0.12203212],[0.2007646,-0.3459293,-0.50306296],[-0.36438587,0.41482344,0.21246466]],[[-0.9344428,0.0134226,0.23387124],[-0.07795933,-0.24448386,-0.24841097],[0.10322628,0.4533753,0.20541333]],[[0.9858357,0.5638333,0.73515016],[0.5106592,-0.01504468,-0.08690476],[0.15510216,-0.27269286,-0.178088]],[[1.0222613,0.12279891,0.4808148],[-0.35098296,0.11615044,-0.10000065],[0.2290466,0.45975992,0.42339915]],[[-0.31189543,-0.44713053,0.8030302],[0.47863996,0.2138139,0.10304419],[-0.0428739,0.37413242,0.48168436]],[[0.35361636,-0.09074613,0.1578254],[-0.26502472,-0.12215889,-1.2272083],[0.3595559,-0.57483816,0.49963117]],[[-1.0408565,-0.42767307,-0.63838357],[-0.15472241,0.14667872,-0.18279906],[-0.9578435,0.04930133,0.09277375]],[[1.065296,-0.33718732,-0.29783604],[0.2207635,0.57280326,0.15247703],[1.0005833,0.39417094,-0.67967725]],[[0.9141897,-0.07320854,0.94493246],[0.89655524,0.19163509,-0.07066178],[1.0842935,-0.4883519,0.66174316]]],[[[-1.68169402e-02,-3.26798797e-01,2.40405858e-01],[5.50641000e-01,-9.37999785e-02,3.17822516e-01],[3.58585656e-01,1.89784437e-01,-1.68018043e-01]],[[5.48911273e-01,-2.42129210e-02,4.27379608e-01],[-2.85263211e-01,4.15823162e-01,-4.97222722e-01],[-6.42232820e-02,6.36297464e-01,3.75114053e-01]],[[1.78146884e-01,-1.65788494e-02,-3.20636928e-01],[7.69153476e-01,2.30431999e-03,5.44881582e-01],[1.26416469e+00,4.40647781e-01,1.43384207e-02]],[[4.13087010e-02,-2.07127556e-01,1.98522806e-01],[1.29228503e-01,1.22493319e-01,-1.26029789e-01],[9.53413360e-03,2.01262422e-02,3.83665979e-01]],[[3.51808608e-01,1.41030937e-01,-1.71619311e-01],[-8.69098127e-01,1.92156792e-01,6.17899373e-02],[-1.29560184e+00,-1.33918202e+00,-6.47490442e-01]],[[-1.28036714e+00,2.37914965e-01,1.70765176e-01],[7.11999059e-01,5.06886542e-02,2.01700985e-01],[-1.94275570e+00,3.63282919e-01,6.21759146e-02]],[[-7.08988369e-01,1.33891255e-01,-5.56816198e-02],[6.56804740e-02,-2.25636333e-01,-2.12408900e-01],[-1.26934850e+00,-7.74650648e-02,6.75582647e-01]],[[-6.06458962e-01,2.47881189e-01,5.86901605e-01],[-3.17935705e-01,2.48351738e-01,9.92281064e-02],[2.26559669e-01,1.33657470e-01,1.24970293e+00]],[[1.17412329e+00,-3.24056745e-01,-8.38497818e-01],[-1.52030289e-01,-4.87034500e-01,-2.14831427e-01],[-9.17769253e-01,-1.86609283e-01,-5.12779832e-01]],[[6.19628966e-01,6.41597658e-02,3.17441881e-01],[4.83919054e-01,-7.90473889e-04,-9.96406749e-02],[-1.85430098e+00,1.38244614e-01,-6.78839386e-02]],[[-1.75052270e-01,-3.13856363e-01,1.51571915e-01],[9.51015800e-02,2.69650280e-01,-3.05387735e-01],[6.30922437e-01,4.23328802e-02,-8.71126130e-02]],[[-3.67639750e-01,-1.59788400e-01,1.97136730e-01],[-2.81430006e-01,-1.35151684e-01,1.93255737e-01],[1.36084402e+00,2.48985469e-01,1.47501543e-01]],[[2.89102137e-01,1.11562282e-01,-2.49822676e-01],[3.28421742e-01,-4.01527882e-02,4.03814539e-02],[6.00839734e-01,-6.78455770e-01,-9.77635145e-01]],[[-6.13465846e-01,3.92044112e-02,8.00932765e-01],[5.45910597e-01,-5.25358260e-01,-2.77416795e-01],[-9.91965711e-01,2.58317113e-01,-2.49405846e-01]],[[-1.77084371e-01,5.01795113e-01,-1.78340152e-01],[-7.74795488e-02,3.01882178e-01,-1.63621500e-01],[4.76465523e-01,1.81634620e-01,-3.07467937e-01]],[[-2.61023700e-01,4.67202216e-01,-3.86496633e-02],[-1.24793962e-01,-5.43155288e-03,4.29892093e-01],[-8.46359670e-01,1.79771587e-01,-2.04814807e-01]],[[-5.70588782e-02,-2.91167051e-01,1.84018150e-01],[4.73032892e-02,-1.44085079e-01,1.68028504e-01],[-8.97503257e-01,3.06511968e-01,2.78695881e-01]],[[5.58432817e-01,-1.85692191e-01,-1.40935439e-03],[-1.61322102e-01,2.30223492e-01,2.13170215e-01],[1.84147906e+00,1.11402385e-01,9.57022727e-01]],[[4.20082301e-01,-2.12801583e-02,-1.67928264e-01],[-3.26910049e-01,-3.32082093e-01,1.76192760e-01],[-5.19729078e-01,-1.21270776e+00,-9.44684207e-01]],[[-9.07306612e-01,-3.28671783e-01,7.51936585e-02],[1.69281483e-01,5.16709447e-01,5.36100388e-01],[-3.02642912e-01,-3.41604263e-01,6.88880235e-02]]],[[[-0.71198857,0.03444358,0.34277818],[-0.35674313,0.04585129,0.42580265],[-0.9478857,0.35305426,0.12402783]],[[-0.9100187,0.20913188,-0.35606882],[-0.71719134,0.2819662,-0.4230712],[-0.6857062,0.24609993,-0.17182983]],[[-0.15277867,0.04710307,-0.11664189],[0.08336183,0.09489428,0.23654957],[0.7080518,0.54917717,0.16685073]],[[0.05467644,-0.16425472,-0.02047105],[0.2714261,-0.22929528,0.15264297],[0.20219861,-0.40436903,0.41853222]],[[0.67310244,-0.02765146,-0.04080635],[-0.28489748,0.28521597,0.32680997],[-0.7168123,0.23624463,-1.0236331]],[[1.0469794,-0.43189976,0.2367959],[0.58520764,0.45198578,-0.3059228],[0.63138866,-0.32776687,0.58282036]],[[0.03202047,-0.41428342,0.09989163],[0.48752168,-0.1909702,-0.06967444],[0.42467985,0.25892782,-0.1748824]],[[-0.52196574,0.19658697,0.01453388],[0.3625991,-0.58345616,0.3575701],[0.2371797,0.3836574,0.3093998]],[[-0.01592349,0.0837414,0.08527043],[0.51300013,-0.3980444,-0.15252574],[-0.96939415,-0.43176508,-1.0386242]],[[0.87685573,-0.1629861,-0.1796729],[0.07224664,-0.02803242,-0.2571679],[1.1888481,-0.4525361,0.16242065]],[[1.7806529,0.3848276,-0.20089108],[0.19226867,0.01314342,0.41950598],[0.59704155,0.97590333,-0.20031506]],[[-0.03740579,0.03366186,-0.23731138],[0.2904908,-0.03769906,0.01239553],[0.13132519,0.12425661,-0.09878725]],[[-0.86196846,0.37627393,0.5404881],[0.22344726,-0.24822003,-0.510151],[-1.13585,-0.6748211,-0.11241646]],[[-0.09085788,-0.09856401,0.38265684],[1.2011074,-0.46130612,-0.71641666],[-0.37688556,0.30420467,-0.6811356]],[[0.04485224,0.11494416,-0.551249],[0.43030837,-0.00524162,0.1846945],[-0.9085285,0.29541647,-0.4974866]],[[-0.8835847,-0.33109814,-0.26979646],[-0.33639055,-0.17571187,0.29348782],[0.41054612,-1.1064823,-0.12872179]],[[0.5345787,-0.47497755,0.05308724],[0.04816547,-0.57413566,0.35708782],[0.50866556,0.37187925,-0.03713997]],[[-1.521549,-0.01140079,0.05190094],[0.22784661,0.18083714,-0.32823792],[0.16087334,-0.04689231,0.13276373]],[[-0.33811834,-0.16120449,-0.2479401],[-0.5947058,-0.67449665,-0.45563692],[-0.5965289,-1.2329417,-0.06866021]],[[2.0329328,-0.18372358,-0.27777895],[-0.18907051,-0.09326684,0.00484231],[0.7592716,0.01145535,-0.62308854]]],[[[7.65251935e-01,-3.71819317e-01,-1.80577353e-01],[4.28633362e-01,-1.15369134e-01,3.83744389e-01],[-6.07978702e-01,-3.21954042e-01,2.97396183e-01]],[[9.39640164e-01,-4.22592819e-01,2.13321939e-01],[-3.03557724e-01,-7.11003006e-01,5.16780674e-01],[-1.74178243e-01,1.09684396e+00,-2.28747919e-01]],[[-4.32321191e-01,-5.45472860e-01,-1.48902774e-01],[3.96490470e-03,3.57772857e-01,4.10116225e-01],[1.25431156e+00,1.15938015e-01,-2.10015014e-01]],[[-1.81197435e-01,1.30019754e-01,2.26364061e-01],[-2.68946260e-01,2.72962838e-01,-1.66365355e-01],[-4.52271163e-01,1.28449500e-01,-3.53823602e-01]],[[-6.38723850e-01,1.06819896e-02,2.72196054e-01],[-3.27803075e-01,-8.31162691e-01,-5.92616677e-01],[-3.85615379e-01,2.02840552e-01,2.43504614e-01]],[[-4.59298670e-01,4.51469183e-01,-8.52327272e-02],[6.02418602e-01,-3.30121331e-02,3.58031452e-01],[8.45120132e-01,5.41717470e-01,-5.60809910e-01]],[[-7.75938034e-01,7.69833982e-01,-3.83786187e-02],[6.97342828e-02,8.45153332e-02,5.24263978e-01],[-3.55540663e-01,1.47789523e-01,-3.41458380e-01]],[[1.38072312e+00,4.54998255e-01,-1.19851775e-01],[-2.53398240e-01,-1.25472933e-01,-7.97448099e-01],[1.45096016e+00,-5.23533523e-01,-1.04717381e-01]],[[-2.82909334e-01,-1.16055632e+00,3.53979230e-01],[6.10852718e-01,1.38863930e-02,9.08026755e-01],[3.47784698e-01,1.30984008e-01,2.50512153e-01]],[[-7.01296389e-01,3.91295493e-01,4.36847448e-01],[-3.01073283e-01,-1.02706003e+00,-7.98820169e-04],[-3.54203701e-01,-3.53873998e-01,3.73656958e-01]],[[-1.04364955e+00,-7.17314422e-01,-5.79864206e-03],[-2.73182094e-01,-8.14567655e-02,2.06742987e-01],[2.34836906e-01,-3.65729071e-02,6.81107223e-01]],[[1.60595566e-01,4.89910275e-01,-4.21360940e-01],[2.04624221e-01,-8.96328986e-02,2.62577701e-02],[3.22131157e-01,1.00634828e-01,-7.42761970e-01]],[[4.99057859e-01,-5.99586844e-01,6.19617701e-01],[-1.67672619e-01,-1.70843288e-01,-5.81612587e-02],[2.67945617e-01,1.86753541e-01,2.86601365e-01]],[[1.20769060e+00,5.75157106e-01,-5.06134629e-01],[-3.20250653e-02,2.48042177e-02,6.22600138e-01],[3.83549482e-01,2.09718734e-01,2.86335021e-01]],[[8.34591150e-01,1.60242543e-02,-2.04769135e-01],[-1.95563495e-01,1.69115320e-01,-2.17574313e-01],[8.79256189e-01,-5.04024208e-01,7.33832791e-02]],[[-5.88810682e-01,-5.96669376e-01,-1.60273731e-01],[9.43723246e-02,5.73803782e-01,-1.12725258e-01],[-1.48158813e+00,3.90637517e-01,1.33610666e+00]],[[-6.17587686e-01,-1.84548169e-01,-4.01985586e-01],[-8.85736197e-02,3.08212310e-01,-4.28510845e-01],[-1.39406729e+00,-6.33406579e-01,3.00742239e-01]],[[7.74343848e-01,1.77719980e-01,3.40129405e-01],[6.39511466e-01,1.67318568e-01,9.41969901e-02],[1.39971256e-01,6.72136843e-02,-6.02639496e-01]],[[-7.94904590e-01,-6.16871357e-01,2.95718670e-01],[7.77283758e-02,-4.85879123e-01,6.64485157e-01],[5.71806431e-01,4.51538920e-01,-4.77063090e-01]],[[-7.59563208e-01,-1.06748603e-01,1.66485831e-01],[4.96005267e-01,1.83786154e-01,-4.07300852e-02],[-6.93151951e-02,4.09919798e-01,3.42291445e-01]]]])
net.state_dict()['block1.3.bias'][:] = torch.tensor([-2.71764e-05,0.00067968725,0.00021845789,0.00025312224,9.5089e-05,1.7194485e-05,0.0011636058,-0.00036583163,-8.020288e-06,0.0008096058,0.00028368726,0.0010144112,-9.36577e-05,3.813175e-05,0.0001659524,0.00010367487,0.0003380769,1.7653016e-05,0.0011824637,8.106779e-05,0.00052839506,0.00011620115,-0.0002227508,0.00026676734,0.00022824627,-7.211449e-06,-0.0012479373,-0.00021604786,-5.8072517e-05,0.00063402863,0.00028622992,0.0010794577,0.0002662824,-0.00040362356,-0.0007971281,0.00028109958,-0.00069546053,-0.00018663167,-4.9241426e-06,0.0005261946])
net.state_dict()['block1.4.weight'][:] = torch.tensor([0.86745065,1.4358399,0.9581177,1.4328018,1.0853136,1.5298712,1.3609889,1.0945032,1.1649889,1.5133598,1.3602134,1.0233911,1.5232666,1.3006265,1.1411545,1.396788,1.1117643,1.278024,1.36428,1.1773709,0.83134955,0.903954,1.0283896,1.1801409,0.9132138,1.7667212,0.9340146,1.52987,1.2447727,0.9880138,0.9267356,2.3647556,0.886537,1.0917757,1.0495266,1.1874033,1.4089735,1.2274789,0.89138997,1.0458544])
net.state_dict()['block1.4.bias'][:] = torch.tensor([0.22665915,-0.55873054,0.19270764,-0.67051953,-0.053724162,-0.67137337,-0.4882673,0.05053102,-0.17268616,-0.25133055,-0.35103974,-0.28759366,-0.8010028,0.04426713,-0.0433365,-0.37284267,0.2885786,-0.1744614,-0.17225404,0.055272106,0.062835835,0.09565918,0.28525966,-0.40573058,0.30418408,-0.599532,0.14410408,-0.33441293,-0.2573016,0.1868441,0.28231597,-1.4341697,-0.0400502,0.06927109,-0.114423595,-0.22286534,-0.43196425,-0.44391048,-0.14795294,-0.06931086])
net.state_dict()['block1.4.running_mean'][:] = torch.tensor([-2.8387926,-0.18517606,2.4023948,2.683225,-1.6068476,-0.88619965,-3.5546415,-0.31026992,-0.9106371,1.6383902,-1.1063815,-1.1012983,-0.5094197,-0.88439643,2.6460896,-2.9907696,1.210371,2.1537063,1.9687068,-0.09334111,0.8098214,-0.9708791,1.7687273,-1.3843741,2.5141509,1.4163532,0.2996387,0.8871781,-3.8503618,-0.23123167,-0.022239078,0.655959,-0.66724426,-3.086087,-0.94971395,2.1059635,2.3031902,0.78162277,-2.4082196,1.5492159])
net.state_dict()['block1.4.running_var'][:] = torch.tensor([27.74123,26.69913,20.409332,47.217773,49.13467,41.545887,29.049564,42.419697,42.97709,57.76239,43.888115,20.293774,37.719845,53.955868,34.1027,41.126556,49.19669,41.73385,39.432087,73.94563,19.160347,18.04962,36.097958,40.742584,47.36243,42.0982,40.88307,40.383625,40.050404,48.954514,30.003836,34.00787,30.446953,40.169914,47.26135,31.758108,41.558662,51.27535,29.52164,33.10258])
net.state_dict()['block1.4.num_batches_tracked'] = torch.tensor(305600)
net.state_dict()['block1.6.weight'][:] = torch.tensor([[[[7.81021416e-01,4.50900137e-01,-3.75862807e-01],[-7.13860631e-01,-6.51935160e-01,-5.52795768e-01],[6.14513278e-01,-3.39686841e-01,-5.70939779e-01]],[[-3.08533758e-01,-8.51997972e-01,1.74394816e-01],[5.33662796e-01,1.24524243e-01,-1.81327108e-04],[-8.13946903e-01,1.39400899e-01,-1.84420049e-01]],[[-1.48053974e-01,5.67733467e-01,-7.64236867e-01],[2.80727863e-01,4.87316877e-01,-8.21409464e-01],[-3.63617331e-01,-1.50897682e-01,3.73208910e-01]],[[1.72004834e-01,4.95079279e-01,2.05164492e-01],[5.58062978e-02,2.13826358e-01,1.61409065e-01],[1.35165066e-01,-1.29961938e-01,1.03021972e-01]],[[4.43310797e-01,-4.11205143e-02,6.33905083e-03],[-1.14434943e-01,7.29721904e-01,-5.03072858e-01],[-3.45500648e-01,-1.03859830e+00,-1.08669364e+00]],[[-3.17530781e-01,4.69273508e-01,-2.26501927e-01],[2.37608850e-01,-4.58838075e-01,3.49881291e-01],[-6.63109601e-01,-2.06554029e-02,-5.27944446e-01]],[[5.06376922e-01,1.51781201e-01,-3.72373968e-01],[-4.81797494e-02,-5.58785737e-01,-1.45998970e-01],[1.83831602e-01,-3.59900862e-01,3.29179555e-01]],[[2.79519349e-01,-6.60816282e-02,1.60448030e-01],[5.42874634e-01,1.91997166e-03,-6.12357110e-02],[-2.39722982e-01,4.81264926e-02,-9.91492346e-02]],[[5.11783548e-02,4.43760008e-02,8.20759177e-01],[1.45426229e-01,-8.55604885e-05,-2.76517197e-02],[1.62328288e-01,-9.02754188e-01,6.00492470e-02]],[[-3.36690336e-01,2.91629732e-01,-1.71292186e-01],[5.02065778e-01,-2.81887442e-01,-1.99185148e-01],[-4.40599769e-01,-4.13524717e-01,5.11734009e-01]],[[2.27166206e-01,-3.12083870e-01,5.75841248e-01],[4.00337577e-01,2.89153993e-01,-3.02015275e-01],[1.03991739e-01,-1.08527505e+00,-9.56557214e-01]],[[2.48054385e-01,9.88701954e-02,-1.16259670e+00],[2.30942339e-01,-5.48308969e-01,-1.56277910e-01],[-7.43022203e-01,-4.43284333e-01,1.55609250e-01]],[[-3.24611932e-01,-1.71055198e-02,-2.43562520e-01],[-1.19677687e+00,-6.84602976e-01,7.80833364e-02],[-8.24105665e-02,-5.07233620e-01,-1.22613680e+00]],[[-1.36718541e-01,1.67409524e-01,-2.71880031e-01],[4.71384674e-02,-1.13175608e-01,2.78850943e-01],[-9.45773423e-02,8.99147540e-02,-2.06104502e-01]],[[-7.57052153e-02,-9.07739252e-02,-7.45059252e-02],[-7.82208264e-01,4.45940197e-01,6.66761324e-02],[4.50960606e-01,-5.74834287e-01,9.37354803e-01]],[[8.77783060e-01,-6.70273066e-01,3.17550041e-02],[-5.05154431e-01,5.58064640e-01,-5.32329798e-01],[-1.00858867e+00,1.87110588e-01,4.12455887e-01]],[[2.47135788e-01,1.26675338e-01,-4.88986492e-01],[-3.89991879e-01,-9.29110199e-02,-5.02783895e-01],[-7.96781242e-01,-3.93033624e-01,2.71563262e-01]],[[-5.43831408e-01,8.86860415e-02,4.44872081e-01],[-5.08449674e-01,-1.14756390e-01,-1.10043116e-01],[-1.90828025e-01,1.36157712e-02,-1.26599491e-01]],[[-2.20225424e-01,-5.71299314e-01,7.15755701e-01],[5.48595488e-01,3.55417728e-01,1.11014044e+00],[-8.41961429e-02,5.89741647e-01,1.73001602e-01]],[[4.60330188e-01,-2.16532797e-01,3.47840972e-02],[-3.83770198e-01,7.34969154e-02,7.90113062e-02],[-3.16281170e-01,5.47972798e-01,4.66253072e-01]],[[5.19690096e-01,-7.22254455e-01,-5.68492198e-03],[-7.46636450e-01,-3.39368552e-01,2.59706825e-01],[3.55522275e-01,1.96250588e-01,1.93318442e-01]],[[7.82265067e-01,-7.12161660e-02,-3.07941318e-01],[-1.99665621e-01,-6.31393269e-02,-5.52655637e-01],[-4.95353311e-01,4.01830524e-01,-3.81491423e-01]],[[1.76751152e-01,1.80622134e-02,-5.72410166e-01],[-4.90421802e-01,1.45291723e-02,3.37871015e-01],[3.45364630e-01,-4.37968411e-02,2.21024096e-01]],[[-1.18120027e+00,6.14203990e-01,-6.37779087e-02],[-1.13423623e-01,-3.05808216e-01,-1.39331251e-01],[4.55541998e-01,-1.39618471e-01,-7.69914925e-01]],[[1.17994277e-02,-4.48269188e-01,4.28013712e-01],[-6.56117380e-01,7.59811848e-02,-6.06109351e-02],[-3.87561888e-01,2.94576973e-01,1.79350942e-01]],[[-2.73578763e-01,-2.34029785e-01,2.45594904e-01],[3.43704224e-01,2.75993615e-01,-1.12460822e-01],[4.46897328e-01,6.43048286e-01,-9.20074046e-01]],[[-5.69203645e-02,-9.78874043e-02,-2.25519195e-01],[-8.26734751e-02,-2.37907246e-02,5.06457351e-02],[6.38741076e-01,-9.36361849e-02,2.23818690e-01]],[[6.08697254e-03,1.67245582e-01,-3.52861166e-01],[1.20495841e-01,5.11907160e-01,8.78955349e-02],[-5.34080982e-01,7.88424194e-01,1.37803292e+00]],[[3.38719308e-01,1.79885864e-01,-2.81041786e-02],[-5.15106976e-01,-5.03508747e-01,3.66092592e-01],[3.91320705e-01,-4.35245559e-02,1.18336149e-01]],[[4.87954952e-02,-4.00896579e-01,-2.62810975e-01],[-7.31941760e-01,-2.51438022e-01,6.90141261e-01],[3.07296813e-01,4.10357118e-01,-8.82678986e-01]],[[5.82094610e-01,-5.48687875e-01,-2.66111881e-01],[2.10671499e-01,-2.73220599e-01,-3.79426837e-01],[2.67706841e-01,-3.59593593e-02,-7.27117807e-02]],[[1.15828192e+00,1.42142907e-01,-1.76615953e-01],[-2.27553502e-01,2.13135004e-01,7.49369338e-02],[-1.79309830e-01,-4.19967920e-01,1.03640592e+00]],[[-4.83900867e-02,-2.86247164e-01,-5.40031493e-01],[7.86234200e-01,-1.21261559e-01,2.44875044e-01],[3.12823713e-01,3.95169169e-01,1.47012681e-01]],[[7.87389800e-02,3.34255487e-01,-4.55674976e-01],[-4.43633229e-01,2.85541236e-01,5.85627019e-01],[6.58342063e-01,4.29619290e-02,5.83644748e-01]],[[3.09884828e-02,4.57933903e-01,2.28225484e-01],[6.96137190e-01,-1.85123861e-01,6.77133739e-01],[-9.14833546e-01,4.70123470e-01,-2.31976524e-01]],[[1.97826490e-01,4.89784867e-01,7.48219013e-01],[4.13849168e-02,-1.07812569e-01,5.73050857e-01],[-1.73804909e-01,-4.12880450e-01,1.52234197e-01]],[[2.44539201e-01,4.17009652e-01,4.39472944e-01],[1.26964569e-01,-5.25538743e-01,3.71268451e-01],[-5.16068339e-01,1.06667340e+00,-1.23726964e-01]],[[5.06272078e-01,-4.63918626e-01,-1.08827077e-01],[-7.54572153e-02,6.09711587e-01,-1.75629146e-02],[-7.48165309e-01,-9.13463950e-01,-2.33798802e-01]],[[-1.08097032e-01,2.46534899e-01,-3.01410019e-01],[3.01319584e-02,1.29317120e-01,2.11996771e-02],[4.81588719e-03,1.03426136e-01,-3.31548661e-01]],[[5.06913364e-01,-1.97447300e-01,9.70150158e-02],[2.31664956e-01,-7.94715524e-01,-1.27938592e+00],[-4.13923234e-01,-1.34332180e-01,-5.55015802e-01]]],[[[1.33188128e+00,3.61442380e-02,9.45240021e-01],[5.37428319e-01,2.56825209e-01,-2.65790939e-01],[5.65276861e-01,-4.16402549e-01,1.55291140e-01]],[[3.95953655e-02,-1.39905661e-01,4.96603668e-01],[-3.58293027e-01,5.36704540e-01,8.84473026e-01],[-3.43346000e-01,-5.43420672e-01,1.92953292e-02]],[[-6.25948682e-02,1.99816659e-01,-5.02003789e-01],[1.68141931e-01,2.88016915e-01,-3.78265530e-01],[1.25034854e-01,-6.89376295e-02,8.01052302e-02]],[[5.57870448e-01,-3.11631471e-01,-5.75341642e-01],[-1.07980445e-02,-1.94625020e-01,-1.17802776e-01],[-4.82468158e-01,1.39203504e-01,-3.53859156e-01]],[[3.04688990e-01,1.42397240e-01,5.09050190e-01],[4.02171552e-01,-4.10546422e-01,-2.59361029e-01],[4.58674610e-01,-4.83770907e-01,2.23919719e-01]],[[1.69698745e-01,1.14565678e-01,6.55221054e-03],[-1.38676330e-01,4.50566798e-01,3.19387972e-01],[2.60532260e-01,1.42153367e-01,-9.00564909e-01]],[[1.47142820e-02,-1.88865647e-01,-3.77126575e-01],[-4.21958983e-01,-4.95629847e-01,3.12022597e-01],[2.12032855e-01,-4.04174775e-01,2.29946271e-01]],[[-6.11569285e-01,-1.31775752e-01,-3.15667957e-01],[4.52847421e-01,2.48498216e-01,-5.71483821e-02],[6.91672266e-01,-5.58706284e-01,-3.39487344e-01]],[[9.75816697e-02,-1.11477382e-01,4.87833589e-01],[-1.03883719e+00,5.04370034e-01,-5.35387695e-01],[-1.21264435e-01,2.98699792e-02,3.25989693e-01]],[[-5.56956649e-01,-1.54465720e-01,2.37297833e-01],[-4.04109024e-02,2.97918320e-01,5.45171499e-01],[2.32055709e-01,2.20970377e-01,6.46397024e-02]],[[-7.75611818e-01,5.69043338e-01,3.95015448e-01],[-6.65563047e-01,2.31879771e-01,3.70714307e-01],[-8.19297791e-01,2.02098876e-01,7.27472186e-01]],[[7.53267229e-01,8.37175921e-02,-3.84599894e-01],[1.49961963e-01,5.52438378e-01,1.19515754e-01],[1.12428498e+00,3.15199822e-01,-2.38159344e-01]],[[-5.35314441e-01,-2.08487406e-01,-2.23228075e-02],[-2.02901274e-01,1.02609120e-01,-6.19601429e-01],[-1.26283512e-01,-1.00547624e+00,-4.13447440e-01]],[[-1.04817506e-02,2.91268438e-01,3.78213674e-01],[3.05915654e-01,5.04398227e-01,3.61847788e-01],[2.04778999e-01,-5.18501922e-02,8.06668937e-01]],[[5.72914004e-01,-2.73388237e-01,5.26385963e-01],[-1.23862438e-01,-4.51144315e-02,4.37557846e-01],[-9.54952016e-02,-5.71979344e-01,9.04518843e-01]],[[-4.97768372e-01,-3.58415782e-01,1.22048035e-01],[-1.63335457e-01,-6.31079003e-02,1.59900993e-01],[3.18946806e-03,6.57639205e-01,4.68448639e-01]],[[-2.87347198e-01,2.47008353e-01,7.07111657e-02],[-5.11127830e-01,-6.89976096e-01,-5.52247047e-01],[-3.66427034e-01,-4.02981415e-02,1.73139513e-01]],[[7.94700861e-01,7.62330234e-01,-2.28335887e-01],[-2.55237490e-01,-5.33537269e-01,-3.80867392e-01],[4.15735602e-01,-6.90376520e-01,3.90323102e-01]],[[3.23623687e-01,3.34104240e-01,-4.44891632e-01],[-5.64907849e-01,-5.07248878e-01,2.55333334e-01],[-7.74312615e-01,-5.07756054e-01,-2.46873081e-01]],[[-4.42551374e-01,3.95164222e-01,-5.96310973e-01],[6.67078197e-02,-3.03189278e-01,-3.98491919e-01],[-1.67917058e-01,-2.47321695e-01,-1.31112352e-01]],[[-2.84178346e-01,1.37135774e-01,8.50395679e-01],[9.06034559e-03,-4.12965208e-01,6.15310192e-01],[1.77655786e-01,3.03403348e-01,6.31877542e-01]],[[4.14199501e-01,-2.33681336e-01,8.27091560e-02],[2.58853376e-01,3.63045223e-02,-4.65886116e-01],[-7.54311800e-01,-2.88569629e-01,2.59272933e-01]],[[-7.36055151e-02,-4.64291632e-01,-1.96555698e+00],[-4.04073671e-02,2.68913567e-01,-4.30856854e-01],[3.12762439e-01,-6.63757250e-02,1.90707445e-01]],[[-3.35985631e-01,-2.25444421e-01,-8.61586988e-01],[-1.11518764e+00,3.20784688e-01,7.36702502e-01],[6.69455767e-01,2.18188897e-01,-2.75510103e-01]],[[-4.91994649e-01,7.88839310e-02,-7.41922617e-01],[-2.52015710e-01,-5.89840055e-01,4.96622026e-01],[-6.45214021e-01,1.74652383e-01,-5.95075309e-01]],[[2.78529972e-01,-1.37646824e-01,3.59309465e-02],[1.32160738e-01,7.19488978e-01,-1.95635542e-01],[4.80000436e-01,-5.14822066e-01,1.78638160e-01]],[[2.94711199e-02,1.71291083e-01,-6.18891239e-01],[-1.20640375e-01,-2.20960304e-02,2.41958782e-01],[-1.25408158e-01,2.28794724e-01,-2.30144694e-01]],[[4.29883897e-02,4.58247811e-01,-6.19026721e-01],[5.34385264e-01,3.82140219e-01,1.88945681e-01],[-3.52563739e-01,-1.39786750e-01,-1.12072182e+00]],[[-1.97740942e-01,-2.36428857e-01,1.09951031e+00],[4.63159606e-02,-9.43015590e-02,1.41374350e-01],[-6.65295541e-01,-5.83497919e-02,5.34478605e-01]],[[5.89045584e-01,6.21585734e-02,4.29317474e-01],[3.67814042e-02,1.83504708e-02,1.08201616e-01],[2.87146032e-01,2.55201995e-01,-5.68448246e-01]],[[-3.94090295e-01,-3.12183022e-01,5.34601450e-01],[1.78345233e-01,-2.37223014e-01,5.69505766e-02],[-6.99015081e-01,-7.12040186e-01,6.04931474e-01]],[[4.92945731e-01,-2.64881253e-01,-4.28502142e-01],[4.97086495e-01,-4.04429942e-01,-7.05201328e-01],[4.03942615e-01,1.68642342e-01,4.35379386e-01]],[[-6.03803515e-01,-2.08622634e-01,5.46149969e-01],[-3.53083834e-02,4.82580870e-01,-3.12722176e-01],[3.53651315e-01,-4.99096289e-02,-2.42247164e-01]],[[-5.46618640e-01,1.17132179e-01,3.19595128e-01],[3.12518775e-01,1.91357926e-01,1.79040372e-01],[-7.87266493e-01,-7.24153295e-02,-5.21638663e-04]],[[8.48367989e-01,1.13061303e-03,-7.78035879e-01],[4.36002046e-01,-5.35250783e-01,5.84137678e-01],[1.57710969e+00,6.97639406e-01,-3.18819642e-01]],[[-7.56285012e-01,-3.20777535e-01,3.40227395e-01],[-5.47097743e-01,-3.28770056e-02,-1.37662198e-02],[-1.05516863e+00,3.24089795e-01,-9.88627002e-02]],[[7.00284466e-02,1.56415731e-01,-4.19192582e-01],[-3.59070867e-01,-4.30367798e-01,5.48275113e-01],[-3.39976251e-02,-8.21243107e-01,-2.01959267e-01]],[[9.21748519e-01,3.10705543e-01,-4.45380747e-01],[1.17271733e+00,-2.98216373e-01,4.57396865e-01],[-4.05110806e-01,-9.71503556e-02,9.83444303e-02]],[[7.97207296e-01,2.35305101e-01,1.74740538e-01],[7.94554472e-01,-8.14343393e-02,-6.90161884e-01],[4.99234080e-01,3.61963272e-01,-7.43351519e-01]],[[-4.15320635e-01,-1.07772097e-01,8.37241262e-02],[3.94171596e-01,-1.86573848e-01,2.40794942e-02],[-1.32991314e-01,-5.84857650e-02,4.94525671e-01]]],[[[-5.80250680e-01,4.41032462e-03,-4.81117964e-01],[5.90098977e-01,1.40576929e-01,-3.89122814e-01],[1.77193210e-02,-2.09431350e-02,1.18726864e-01]],[[1.49513543e-01,1.58062115e-01,2.39283562e-01],[3.15453261e-02,6.10703044e-02,-1.02827683e-01],[-2.27271542e-01,9.18544978e-02,3.02154064e-01]],[[-1.98568970e-01,7.22392857e-01,-4.68185037e-01],[4.34347153e-01,1.47223741e-01,-7.01018333e-01],[-4.59827513e-01,5.43798447e-01,-9.93435621e-01]],[[5.39548576e-01,3.37715834e-01,-6.95878327e-01],[-1.61378771e-01,1.93828121e-01,-3.12331051e-01],[3.66418183e-01,-1.94048777e-01,-7.49927700e-01]],[[6.90383418e-03,-4.22211617e-01,-2.99328834e-01],[8.62419784e-01,7.63264894e-01,-1.03117444e-01],[3.51692438e-01,-7.20601678e-01,3.82313073e-01]],[[1.58178397e-02,1.02661297e-01,3.46982889e-02],[-2.24869892e-01,-4.97027636e-01,5.85044920e-01],[-8.14347208e-01,-5.22240400e-01,-3.21648628e-01]],[[-5.76042056e-01,8.12656999e-01,-6.95732176e-01],[5.31010330e-01,9.16932106e-01,-3.12194943e-01],[6.81766093e-01,-3.59961450e-01,1.56571735e-02]],[[2.06085164e-02,-6.46517932e-01,-4.07494098e-01],[1.73750743e-02,-1.37567837e-02,1.00219214e+00],[-4.45168585e-01,8.88459325e-01,-3.86661058e-03]],[[3.92447889e-01,-4.11610186e-01,3.14642876e-01],[-2.96493381e-01,-2.02712357e-01,7.59627521e-01],[2.86530703e-01,2.37037405e-01,-4.04275417e-01]],[[3.59817296e-01,-2.35344637e-02,2.38017485e-01],[2.78370976e-01,3.98820966e-01,8.29097152e-01],[4.32286382e-01,-2.47019544e-01,-1.52873784e-01]],[[1.02250719e+00,-3.45495701e-01,9.45595801e-02],[1.37158602e-01,2.85218656e-01,8.42024833e-02],[3.17964286e-01,-9.59484816e-01,-4.78658766e-01]],[[-1.51653737e-01,1.41498983e-01,2.10040778e-01],[2.01280918e-02,-4.77386862e-01,-3.92012268e-01],[3.22295666e-01,-3.95614072e-04,1.09179175e+00]],[[1.52723640e-02,-4.66196239e-01,-1.15002024e+00],[4.90190536e-02,2.08290711e-01,1.81545317e-01],[-6.59273982e-01,-9.23793912e-01,1.10660747e-01]],[[2.02504978e-01,-3.60556319e-02,-9.93220583e-02],[-1.35824010e-01,-2.13404968e-01,3.72931898e-01],[6.46317285e-03,8.22756410e-01,6.61796868e-01]],[[2.47174129e-01,5.92609763e-01,2.21151441e-01],[2.62315959e-01,-2.61282951e-01,2.04489559e-01],[-5.53013265e-01,2.53009856e-01,-1.14959991e+00]],[[5.40039241e-01,-2.26686135e-01,-3.94167900e-01],[5.91915429e-01,-1.76730812e-01,-8.63830507e-01],[7.07339466e-01,3.77021760e-01,-2.00572461e-01]],[[-1.48069859e-01,1.21994361e-01,2.24615812e-01],[-3.62799972e-01,5.71680427e-01,-4.70141411e-01],[-1.17868865e+00,-2.69543529e-02,-5.08941293e-01]],[[3.56010705e-01,2.59549111e-01,-1.83969941e-02],[1.22754566e-01,-2.89974630e-01,1.41693532e-01],[5.45147434e-02,1.60312906e-01,3.55314702e-01]],[[7.77881444e-01,1.68119699e-01,-3.35656166e-01],[-1.01578407e-01,-2.72056878e-01,-4.29688357e-02],[2.45858785e-02,-3.07796299e-01,4.46331263e-01]],[[5.98930009e-02,-1.12772143e+00,-6.91383183e-01],[1.30836681e-01,3.06619346e-01,-1.26850503e-02],[7.99470901e-01,2.34850943e-01,-9.65712547e-01]],[[3.23858917e-01,-5.74928403e-01,1.60966247e-01],[3.28010499e-01,-1.23962380e-01,-3.54579426e-02],[-2.52705544e-01,-1.87332612e-02,1.52554899e-01]],[[-4.13366109e-01,-4.56214137e-02,-1.52934968e-01],[-2.66053677e-01,2.44361550e-01,1.22019507e-01],[7.60415420e-02,4.40887988e-01,-4.67867047e-01]],[[-3.45270157e-01,7.31839657e-01,5.16195536e-01],[-1.34829193e-01,-3.17237258e-01,5.24255991e-01],[-7.14178443e-01,5.30871861e-02,6.27364635e-01]],[[-3.13842386e-01,7.23338723e-02,-4.27643299e-01],[-3.43126953e-01,4.09238428e-01,-4.92228240e-01],[3.58578153e-02,-1.32103845e-01,3.88668627e-01]],[[-3.24514806e-01,-1.14490949e-01,-5.05884528e-01],[3.99622709e-01,-2.59811670e-01,-2.68185884e-01],[3.07941079e-01,-1.69820234e-01,-2.78293174e-02]],[[-1.13093652e-01,-1.37677506e-01,4.11523074e-01],[1.18358001e-01,1.03261602e+00,8.46650481e-01],[8.21264625e-01,2.96695203e-01,-3.05262536e-01]],[[3.72794509e-01,4.16845620e-01,-5.72434843e-01],[1.06902647e+00,-7.45605946e-01,-3.19956779e-01],[1.19862318e-01,-1.97614565e-01,-2.77774096e-01]],[[8.76168460e-02,-1.95535153e-01,-8.26332495e-02],[3.81303996e-01,-4.44886982e-01,8.73067617e-01],[-4.87074733e-01,4.79548603e-01,1.48245111e-01]],[[5.00429213e-01,5.19537807e-01,1.13468111e-01],[9.72375929e-01,-5.11327907e-02,-2.37136409e-01],[-7.42103755e-02,-2.65848935e-01,-3.21922958e-01]],[[-3.11860055e-01,-3.34596559e-02,5.15910029e-01],[-7.84399688e-01,1.35591626e-01,-2.06408314e-02],[1.37782255e-02,2.32225522e-01,-2.39420369e-01]],[[8.02337408e-01,-4.24798429e-01,-1.02754188e+00],[5.75351179e-01,-3.45161319e-01,-1.19625814e-01],[-8.94436538e-01,2.17746347e-01,-9.18169498e-01]],[[-9.06188965e-01,-1.23153761e-01,1.07911378e-01],[-3.22047830e-01,2.87127137e-01,1.08115442e-01],[2.48927221e-01,6.65240347e-01,-5.25942922e-01]],[[-1.34855062e-01,-8.50031912e-01,-1.05898535e+00],[6.74362242e-01,-3.04254368e-02,-5.23187935e-01],[-6.09168589e-01,-6.28389567e-02,-9.46847677e-01]],[[-4.69076425e-01,-3.60457748e-01,3.15629423e-01],[4.41708863e-02,4.82653379e-01,4.14310306e-01],[2.13094056e-01,-4.84002352e-01,-1.99602619e-01]],[[3.68556827e-01,-5.73585741e-03,-1.70154825e-01],[1.07858725e-01,1.42227873e-01,1.81950986e-01],[2.30548248e-01,1.40387982e-01,-1.69553280e-01]],[[2.77151585e-01,3.63809586e-01,4.93903846e-01],[1.03033565e-01,2.90162172e-02,-8.11170995e-01],[-3.56009841e-01,-3.23728442e-01,8.19418490e-01]],[[-5.80951273e-01,1.46560341e-01,-5.82789958e-01],[-6.08194351e-01,-3.05234015e-01,-2.89786994e-01],[-1.88080564e-01,-2.43726313e-01,1.78393632e-01]],[[4.19240683e-01,7.52793074e-01,-2.67891914e-01],[4.17777747e-01,1.74721524e-01,4.50110763e-01],[-3.01585615e-01,5.72592206e-02,-1.74717188e-01]],[[-1.53212219e-01,-2.98946738e-01,-1.90253168e-01],[2.12713107e-01,-3.76898974e-01,-1.03159919e-02],[4.20864582e-01,-3.52651060e-01,4.46499288e-01]],[[4.06480759e-01,-2.48058066e-01,5.06302238e-01],[8.74622762e-02,6.98518381e-02,-2.28243768e-01],[-4.46508259e-01,-9.54860747e-02,9.82062370e-02]]],[[[-3.42420250e-01,-1.36073366e-01,-2.52670139e-01],[-1.80789769e-01,-5.19417167e-01,-4.54263955e-01],[-1.18455542e-02,1.96596637e-01,-1.28828704e-01]],[[-5.71095645e-01,-4.88103300e-01,-1.99691400e-01],[1.75673544e-01,-1.27129734e-01,2.82123178e-01],[3.24527204e-01,3.33322495e-01,-2.34295905e-01]],[[-9.14924815e-02,6.82218075e-01,-2.20205948e-01],[-7.11611032e-01,-3.76223843e-03,-2.36393437e-01],[-3.28650802e-01,1.20243952e-01,2.82612324e-01]],[[5.95663488e-01,2.68026412e-01,5.48560917e-01],[-3.36664051e-01,-4.79384065e-01,4.32491213e-01],[-6.09172225e-01,-4.39843327e-01,6.47682250e-01]],[[6.25866890e-01,1.12910822e-01,-1.12491474e-01],[-3.83382738e-01,-6.47943020e-02,-7.58096516e-01],[5.54370463e-01,3.47899079e-01,-1.46473080e-01]],[[-8.33541393e-01,-2.15413988e-01,5.85526288e-01],[-1.85165465e-01,4.79310215e-01,4.83475141e-02],[-6.78528011e-01,-6.79863170e-02,-5.57636380e-01]],[[-1.48541957e-01,-4.79447335e-01,1.20363443e-03],[-1.38073340e-01,6.12043679e-01,4.83261853e-01],[-2.76438922e-01,1.01509368e+00,-4.06369090e-01]],[[8.39606747e-02,2.94702679e-01,-1.00051291e-01],[-6.05099723e-02,3.03393036e-01,-3.34005743e-01],[-2.34158903e-01,6.61696672e-01,-3.78206044e-01]],[[1.03645541e-01,8.41123998e-01,-1.98090419e-01],[1.02363884e-01,-1.27383068e-01,-2.93266326e-01],[4.74041386e-04,1.96943149e-01,1.45861298e-01]],[[2.04703838e-01,8.02501887e-02,-5.74188948e-01],[5.73924363e-01,3.95909607e-01,4.07258375e-03],[2.71236151e-01,-6.83781564e-01,-1.37244061e-01]],[[-5.85057020e-01,4.16611396e-02,2.61104368e-02],[4.78410758e-02,-6.81888044e-01,-3.13523263e-01],[-5.74262321e-01,1.54377773e-01,-7.49261856e-01]],[[2.62836009e-01,3.73558283e-01,-2.74275899e-01],[1.01776481e+00,-3.62028837e-01,-9.92968202e-01],[4.30193335e-01,1.83314517e-01,-1.62648188e-03]],[[-5.69255874e-02,9.48293269e-01,1.54977381e-01],[-4.70601022e-01,1.81624576e-01,7.33446470e-03],[-1.88280746e-01,-4.99228895e-01,-1.06551334e-01]],[[-5.00036895e-01,-1.38696164e-01,-5.29802263e-01],[-8.27373385e-01,1.24768637e-01,7.36836076e-01],[-2.84860551e-01,-2.34002605e-01,-2.21574772e-03]],[[1.77908376e-01,-3.13418329e-01,-3.08919132e-01],[-4.49491173e-01,-2.20057935e-01,-3.49193960e-01],[-1.16789830e+00,-5.58643825e-02,7.58663043e-02]],[[4.10293728e-01,3.03799771e-02,-1.61300138e-01],[1.12381354e-01,6.03877723e-01,-2.63612509e-01],[-1.09852031e-01,-1.32361427e-01,-1.31101742e-01]],[[3.10621321e-01,6.67574048e-01,-6.66871428e-01],[2.11751491e-01,9.52010930e-01,-6.79930627e-01],[2.79734582e-01,4.12815690e-01,-7.51251131e-02]],[[-2.95938075e-01,2.30101243e-01,1.02356267e+00],[-7.56951272e-01,2.93409437e-01,-6.79095834e-02],[9.71849203e-01,-2.93326348e-01,5.71617603e-01]],[[-4.04705219e-02,3.00492384e-02,2.43431598e-01],[-5.12560844e-01,-2.34774590e-01,6.60768524e-02],[9.02953267e-01,2.90078312e-01,-3.41127962e-01]],[[-4.87716764e-01,-6.45218194e-02,-3.12615037e-01],[7.11541295e-01,4.74668503e-01,-2.98539433e-03],[6.42152354e-02,-9.07701850e-02,1.41357410e+00]],[[1.26673013e-01,2.30247945e-01,4.29863691e-01],[-4.51714396e-02,-6.57746270e-02,-2.31334269e-01],[-6.29787683e-01,2.10220292e-01,-3.18946958e-01]],[[-3.27298671e-01,2.35143229e-01,-4.97512400e-01],[-4.89739448e-01,2.35156685e-01,1.78695455e-01],[-4.50114250e-01,2.81072140e-01,-7.66685843e-01]],[[1.14726734e+00,2.06229463e-01,1.15727901e+00],[-5.05977392e-01,1.76521227e-01,2.47862771e-01],[1.13266073e-01,3.21007334e-02,6.87323272e-01]],[[-1.48711070e-01,-7.50149861e-02,2.96032995e-01],[4.74358886e-01,1.51809797e-01,2.36418158e-01],[-3.11635807e-02,1.47640571e-01,3.78159225e-01]],[[1.21770259e-02,2.37243429e-01,2.14857355e-01],[5.32938361e-01,-2.20488518e-01,2.17405036e-02],[2.96874702e-01,3.68545324e-01,7.36312330e-01]],[[-6.11318089e-02,3.14048350e-01,-1.98969264e-02],[3.96801978e-02,-4.29566681e-01,-3.29026610e-01],[6.37299299e-01,5.14410794e-01,-5.84164023e-01]],[[-5.94195127e-01,-5.06341755e-02,-4.64322604e-03],[6.27151906e-01,-1.01469256e-01,-1.36795163e-01],[-9.22134578e-01,-7.30634570e-01,1.14076853e-01]],[[1.42801195e-01,-2.68020004e-01,2.51842260e-01],[2.59114653e-01,6.44774795e-01,5.02434373e-01],[-5.62915504e-01,-4.99183893e-01,1.81090489e-01]],[[1.04521349e-01,3.54262561e-01,-4.94570494e-01],[-3.39355439e-01,1.82275832e-01,-1.44626752e-01],[3.75064999e-01,3.54395926e-01,-8.86754215e-01]],[[2.52745032e-01,-4.40833688e-01,7.83452511e-01],[-5.84092855e-01,9.04067233e-02,-1.28158033e-01],[3.40978026e-01,3.87628645e-01,1.72195435e-01]],[[8.40867385e-02,-1.97907954e-01,-1.30812615e-01],[-6.63128257e-01,1.68000922e-01,2.38042295e-01],[-3.60803783e-01,3.94913107e-01,-5.30923195e-02]],[[1.28317162e-01,-4.42642123e-01,-4.13359344e-01],[8.67000967e-02,-6.39735639e-01,8.71333241e-01],[-3.26804608e-01,3.73411447e-01,-2.83656150e-01]],[[-2.33997479e-01,-3.06137949e-01,-1.64967462e-01],[-4.41717178e-01,2.30001837e-01,-7.49216259e-01],[-1.36005148e-01,2.12830789e-02,4.23744529e-01]],[[-1.09224349e-01,-3.65160895e-03,-5.20471573e-01],[2.10248604e-02,-8.66504684e-02,-4.38790828e-01],[3.35236192e-01,-1.75052404e-01,-1.34747982e+00]],[[4.27534617e-02,4.53884482e-01,1.90236479e-01],[2.07995787e-01,1.94356591e-01,-1.17781007e+00],[-3.33841622e-01,2.90725112e-01,-2.13521644e-01]],[[-1.09468389e+00,-1.23794012e-01,-4.71274346e-01],[-6.54629946e-01,1.64171249e-01,8.47038686e-01],[-4.49228473e-02,-1.77436143e-01,-2.12941527e-01]],[[-8.00838530e-01,-2.45760456e-01,2.92977422e-01],[-9.11232293e-01,2.59334564e-01,-8.90246704e-02],[-2.23761380e-01,3.12851757e-01,1.71134710e-01]],[[-1.37739703e-02,4.66590732e-01,-8.26963544e-01],[4.45617318e-01,2.36978635e-01,1.91446260e-01],[-5.33351079e-02,-1.41526774e-01,-5.52020788e-01]],[[-6.52382672e-01,-1.13161016e+00,2.11991638e-01],[-7.51690120e-02,-3.53525221e-01,2.18948632e-01],[-2.18757600e-01,1.43453494e-01,1.82056967e-02]],[[5.58037817e-01,3.12213510e-01,2.32821047e-01],[1.58788860e-01,-4.49440718e-01,-2.59572029e-01],[-4.91823941e-01,-1.13977522e-01,-8.82413566e-01]]],[[[-0.10652875,-0.37354895,-0.56094724],[0.07672546,-0.8366651,0.00334099],[0.29907143,0.14104845,0.31967697]],[[0.48034003,0.55590844,0.2778296],[0.6314052,-0.2498387,-0.00328126],[1.035014,-0.6083526,0.54697573]],[[-0.08377445,-0.17475866,0.27928695],[0.09369733,0.5429312,0.11407221],[-0.0850441,0.10796572,0.6120726]],[[0.32684758,1.0448091,0.73942786],[0.07213227,0.3670512,-0.48206335],[0.6390781,-0.43839177,-0.6680256]],[[-0.577651,0.1078328,0.3102193],[-0.6496699,0.19076794,0.5976779],[-1.0389074,-0.06557361,-0.5462375]],[[-0.4982001,0.30939376,-0.33123487],[-0.30354813,-0.9785634,0.39547512],[-0.45015928,-0.46789467,1.0223207]],[[-0.89011127,-0.2459602,-0.37093726],[-0.07759054,-0.73660374,-0.07597448],[-0.02583675,-0.5388026,-0.9712057]],[[0.01489952,0.09364702,0.31180042],[0.12865008,-0.3075905,0.4813425],[0.38683403,0.65833414,-0.21991402]],[[-0.854772,-0.11187453,0.43490276],[-0.20307972,0.41745377,0.28903002],[-0.31249008,-0.03175264,0.7967459]],[[-0.3972837,-0.09887565,0.40279192],[0.3300478,0.24043585,-0.2970942],[0.53695434,0.65793633,0.09143853]],[[0.64227676,-0.55148625,-0.38906708],[1.1366029,-0.5119992,-0.37658486],[1.148985,0.29822966,0.0799614]],[[-0.04234549,-0.01234488,-0.15658914],[0.06358588,0.2832208,0.4913025],[0.05745218,-0.36760744,-0.00963607]],[[-0.34114948,0.1896613,0.06090352],[0.12880886,0.02466155,0.1014825],[-0.14633663,-0.10733644,0.51949817]],[[0.2075862,0.19213529,-0.4430414],[0.33408174,0.35472178,-0.28808394],[1.0263749,-0.31452236,0.01438112]],[[-0.07368145,-0.6623877,0.32832685],[0.16322798,-0.4795395,0.65057415],[0.11515809,-0.13613686,0.98693657]],[[0.14686505,0.23794898,-0.4212827],[0.23813288,0.07079069,-0.77849513],[0.1924616,-0.5568922,-0.69800586]],[[-0.18166009,-0.71927154,-0.22231293],[0.1532452,0.44275823,-0.2368668],[0.6835405,0.48250368,0.58766884]],[[0.16004173,-0.27585843,0.00429627],[-0.14686194,-0.2022026,0.2152048],[0.79862845,-0.16751583,-0.22231634]],[[-0.19158845,-0.18685786,0.8519476],[0.26805437,-0.05080744,0.11295772],[-1.0405706,0.65747106,0.29942834]],[[0.00139363,-0.03493473,-0.23379487],[0.08798098,0.0312135,-0.37201515],[0.12231628,-0.3753761,0.47955248]],[[0.5364413,-0.15822393,-0.04366954],[0.5279252,-0.09536199,-0.08137407],[-0.04384616,0.39941558,-0.58433485]],[[-0.5269399,0.2529234,-0.7341767],[-0.19874997,-0.24047205,-0.11769988],[-1.168604,0.24818678,-0.15362532]],[[-0.03739359,0.24609861,-0.40679276],[-0.13142931,-0.22415823,0.23769906],[0.5597104,-0.17119679,0.36017054]],[[0.7811366,0.20683488,-0.04267177],[0.9509098,0.04686806,-0.7586351],[0.4940263,-0.45691162,-0.7741403]],[[-0.00694158,-1.3270105,-0.27225444],[-0.55422765,-0.05130777,-0.3775553],[0.21432309,0.6882462,-0.48353773]],[[-0.1584198,0.5019777,-0.06025942],[-0.5418241,0.79149497,0.3491665],[-0.33701444,0.16769788,0.5707605]],[[-0.83469355,-0.44345033,-0.3511272],[-0.262282,-0.08148447,-0.492031],[-1.1615404,-0.13906577,0.35816818]],[[0.4036894,-0.4585618,-0.056678],[0.00139534,-0.23223844,0.56981826],[0.16315083,0.5428581,0.04884555]],[[0.2218791,0.15198989,0.3889644],[0.09518617,-0.28646398,-0.38468277],[0.33573294,-0.06793241,0.6607121]],[[0.45577145,0.05133384,-0.07064158],[0.34819028,-0.42631888,-0.11086809],[-0.20592949,-0.91504955,0.53313404]],[[0.35708374,0.01622466,-0.17325002],[1.2288623,-0.77826256,-0.25540808],[0.01128013,-0.2293787,0.09537325]],[[0.3628065,0.3247455,-0.22672054],[-0.7180843,0.26876333,-0.7975285],[0.18930648,0.26668575,-1.3392762]],[[-0.4796157,-0.28474674,-0.4313148],[0.4666331,-0.46077994,-0.24742566],[-0.38915232,0.19986725,-0.52695984]],[[-0.03691726,-0.46760273,-0.02956336],[-0.10828578,0.18564944,0.5025485],[-0.3478252,-0.00921409,0.48973304]],[[-0.5078236,-0.17862554,0.04118461],[-1.0957474,0.6924244,-0.39132538],[-0.42218435,-0.44599742,-0.3801055]],[[-0.07782145,0.32395175,-0.35724738],[-0.7263307,-0.20037241,-0.41275173],[-0.19102238,-0.13641554,0.35916346]],[[0.43045032,-0.4293995,-0.20388797],[1.2306987,0.19226778,0.06622799],[-0.4820017,0.30975604,0.71731335]],[[-0.802982,0.2595794,0.13138218],[0.38782468,-0.22059351,-0.14184384],[-0.38321915,-0.21614866,-0.6304719]],[[0.26294497,-0.30143207,-0.42640036],[0.5883596,-0.15944897,-0.24852899],[0.31079006,-0.08737821,-0.26995653]],[[-0.23056893,0.19646345,-0.6154687],[0.7224339,0.2958031,0.2634232],[0.294125,0.16985711,0.21680704]]],[[[-0.8960394,-0.4542613,0.7022972],[-0.43286076,-0.26794183,0.37583622],[-0.45610368,-0.4338384,0.49980843]],[[-0.01302521,0.41612443,-0.70620877],[-0.04668887,-0.5628739,-0.7928095],[0.4888719,0.22735429,0.21206418]],[[-0.9339235,-0.05427146,0.4332096],[-0.5221389,-0.5526754,-0.45731112],[-0.14286403,0.5646606,0.20822409]],[[0.2835153,0.45161217,-0.42423776],[0.5300684,-0.5071543,0.512762],[-0.8703462,-0.6464827,0.3392008]],[[0.20305872,0.4569198,0.16528974],[-0.79429024,0.37450218,0.02270919],[0.87317705,0.02585056,-0.02449681]],[[-0.07656422,-0.27826267,0.42807156],[0.05555901,-0.45364982,0.14976007],[-0.2491396,-0.67980266,-0.4782275]],[[-0.30746874,-0.1471369,-0.69199276],[0.17973605,0.8249195,-0.8840924],[-0.23954663,-0.17582604,-0.86661196]],[[0.22412778,0.1906849,-0.64573556],[-0.26721087,0.01806576,0.6974984],[0.06987274,-0.49547738,0.37264073]],[[-0.03167352,0.66279435,0.07882135],[-0.6309023,-0.19532737,-0.16215761],[0.29297346,-0.45554608,0.9259306]],[[0.54631096,0.14713998,-0.04771376],[0.6777594,-0.2805582,-0.15086761],[0.9610418,0.13416347,0.20025626]],[[-0.0619818,-0.92071134,-0.73619884],[-0.7367801,-0.52067363,0.1832817],[-0.8668228,0.62020344,0.18402216]],[[-0.54179376,0.62883925,0.03608144],[-0.7179858,0.5229709,-0.71411973],[-0.3963653,0.28885368,0.22791547]],[[0.1293344,-0.34647873,0.2283426],[0.20632161,-0.86137867,-0.2183709],[-0.06484369,0.02116608,0.597943]],[[0.0638419,0.1086534,-0.13542181],[0.19914037,0.281128,-0.02847289],[0.00826328,-0.17059037,0.00638601]],[[-0.4249346,-0.7025318,0.21622601],[-0.23778683,-0.27552262,0.44048584],[-0.43085736,0.06982435,-0.5821563]],[[0.48171267,-0.06265877,-0.08204306],[1.2060195,-0.5617915,-0.08266731],[1.1430092,-0.42463997,-0.40849662]],[[0.23879361,0.00129386,-0.3918202],[-0.07017076,-0.11184489,0.00986719],[0.04483138,0.32992396,-0.09872075]],[[0.45537654,-0.30312875,-0.5562673],[-0.31612414,-0.02959849,-0.22733104],[-0.38638481,-0.5060027,0.21549481]],[[-0.09284266,0.21668448,0.24860528],[0.9420599,0.267882,-0.3358037],[-0.07065484,0.3269205,0.9367949]],[[-0.02904005,0.08136521,-0.5844624],[0.05998545,0.33815768,0.04622133],[0.41950408,0.31350908,0.17665902]],[[-0.7242543,0.33606315,0.19141974],[-0.74043226,-0.01426612,-0.16785648],[-0.00592747,-0.0878751,1.0127652]],[[-0.29068652,0.23854028,0.35316238],[0.10859718,-0.39417955,-0.07138113],[0.3151196,0.07765389,0.12002499]],[[-0.12522694,-0.34081346,0.25979224],[0.86279494,0.07670357,-0.11622001],[0.78038996,-0.4267363,0.2285257]],[[0.6498588,0.30844867,-0.92254436],[-0.00630398,-0.63813686,0.2025919],[1.1348246,-0.05487222,0.5198072]],[[1.2895507,-0.1176769,-0.21664008],[0.7918907,-0.28623882,-1.1646625],[0.05722334,0.09343947,0.3094375]],[[0.39126948,-0.0902715,0.16948888],[-0.802368,0.70639473,1.0332657],[0.15138504,0.2231235,0.37232816]],[[-0.31465572,-0.06152617,0.2029661],[-0.0987647,-0.20195581,0.18094258],[0.42606422,-0.46159613,-0.17863144]],[[-0.45794728,0.31625947,0.4223727],[0.76728266,0.6086074,0.5081943],[0.42597535,0.16807134,0.09172139]],[[-0.13871643,-0.32245317,0.14232108],[-0.5800905,-0.5136718,0.09775376],[-0.60987073,-0.25264132,-0.5442249]],[[0.44113013,-0.32004645,0.10960564],[-0.12895893,0.45101708,0.30241144],[-0.00962217,0.6985975,-0.0579897]],[[0.7975473,0.53693986,-0.35473397],[0.05463911,-0.29653767,-0.59321207],[0.22425503,0.05910619,-1.1046947]],[[-0.3321767,-0.57672316,0.6691289],[0.01215805,0.7085179,-0.66845995],[-1.1025441,0.601407,0.51848245]],[[0.7495661,0.18697107,-0.8270959],[0.6753914,-0.2547538,0.13992436],[0.12665753,-0.24584724,0.55223376]],[[0.6257378,-0.42837363,-0.35700768],[-0.6476581,0.279145,0.5617638],[0.3385569,0.44063866,0.52717304]],[[0.2879541,-0.13284364,-0.34319517],[0.24325477,0.01298726,-0.0995069],[0.02148931,-0.05683333,-0.15661287]],[[0.23569342,-0.02298858,0.02605886],[-0.17860436,-0.40772405,0.17754863],[-0.28515193,1.0756305,-0.5379606]],[[-0.6259767,-0.41291893,-0.79151654],[-0.60657334,0.20760652,0.73563194],[0.2147169,-0.1551825,-0.48904273]],[[0.023519,-0.43282476,-0.19009742],[-0.34412792,0.8039457,-0.26898783],[-0.03243823,-0.17732869,-0.5874278]],[[-0.146811,0.47533286,-0.11362737],[0.01536113,-0.15508582,-0.07176345],[0.09600503,0.92629385,0.0089853]],[[0.21881223,-0.9685651,0.05469223],[-0.31042367,-0.335246,-0.13885894],[0.30221644,-0.6854332,0.2823882]]],[[[7.14507580e-01,4.05664027e-01,2.12770596e-01],[2.26066202e-01,1.15213946e-01,-6.84060395e-01],[-6.25485480e-02,6.85554445e-01,9.85170081e-02]],[[-1.05285811e+00,-7.26469085e-02,-5.13111889e-01],[-4.92576629e-01,3.79128277e-01,-1.88662544e-01],[3.46572429e-01,4.65338081e-01,3.47135991e-01]],[[-4.01515216e-01,-1.69490010e-01,-1.15209110e-01],[1.24452226e-01,5.53532615e-02,-6.79323018e-01],[-1.56985149e-01,-3.75079550e-02,-2.28822649e-01]],[[-1.90129459e-01,-1.93115667e-01,4.13349755e-02],[-4.88614291e-01,-3.54065448e-01,-2.67318729e-02],[-1.68669224e-01,-1.13724545e-01,-1.17009610e-01]],[[-1.21128358e-01,8.57522115e-02,-2.62987008e-03],[4.79890466e-01,1.03264842e-02,-4.99186158e-01],[-2.54058298e-02,-8.52406621e-02,-1.26028752e+00]],[[3.67978215e-01,2.12159947e-01,-3.45751464e-01],[-4.52305526e-01,4.11015600e-01,4.03548390e-01],[-7.07797706e-01,-1.49267769e+00,-8.99186313e-01]],[[-4.83082712e-01,-3.87438804e-01,-1.31733073e-02],[8.96648020e-02,-3.97351533e-01,-3.23950201e-01],[-7.78131008e-01,7.36380219e-01,-7.68318236e-01]],[[-4.51635629e-01,3.25539976e-01,-1.89213187e-01],[5.51564693e-01,-2.45084196e-01,-7.32707918e-01],[-3.51861119e-01,-3.10526520e-01,-6.07678831e-01]],[[-3.01049381e-01,-3.74846846e-01,2.99582303e-01],[-7.26416349e-01,-2.96449661e-01,3.00020855e-02],[4.52715188e-01,1.75109655e-01,4.27699834e-01]],[[-3.90074641e-01,-1.97136812e-02,7.06666350e-01],[-2.57863700e-01,8.88552889e-02,-4.44225699e-01],[-9.05219853e-01,6.26036525e-01,-3.88655931e-01]],[[-4.77201015e-01,-2.16438428e-01,3.61445397e-01],[-4.77314860e-01,5.13237774e-01,-3.16944599e-01],[-5.33308148e-01,-5.83208442e-01,-1.01797354e+00]],[[2.41912212e-02,-3.25284809e-01,2.53372252e-01],[-3.10018752e-03,-3.45512271e-01,-1.21996021e+00],[1.09491318e-01,1.36006568e-02,-7.44820654e-01]],[[1.94089815e-01,-3.90450656e-02,-3.02215397e-01],[-1.31607026e-01,3.19288701e-01,-1.68274447e-01],[-6.90808415e-01,2.22528204e-01,3.65062118e-01]],[[2.10640997e-01,-8.23112503e-02,9.80042964e-02],[1.88923091e-01,1.74784273e-01,1.32730559e-01],[4.95601624e-01,-6.14422917e-01,-1.51235417e-01]],[[-1.17029510e-02,-3.87914211e-01,3.10542397e-02],[5.95071793e-01,2.85488486e-01,1.66947156e-01],[-2.03145444e-01,2.81723469e-01,-1.53282225e-01]],[[-2.35551089e-01,-2.55208284e-01,1.76676679e-02],[-7.21102208e-02,3.67045224e-01,-8.07122231e-01],[1.58665031e-01,2.51944810e-01,-5.18829048e-01]],[[-2.79357076e-01,4.02746022e-01,-2.90614128e-01],[-2.48852059e-01,6.79338723e-02,-5.02352655e-01],[-6.94192573e-02,-3.39817226e-01,5.83024681e-01]],[[5.20302057e-01,-5.10368347e-02,-5.28207839e-01],[-2.51628458e-04,5.74986756e-01,-7.55104661e-01],[3.93991262e-01,6.33135736e-01,-6.60218537e-01]],[[-5.31353414e-01,-2.20397130e-01,7.35824287e-01],[1.25341535e-01,3.80618393e-01,1.45678729e-01],[-2.25824460e-01,8.57008025e-02,-7.86409616e-01]],[[-2.25636527e-01,-5.65795600e-01,3.33505392e-01],[4.14190888e-02,4.76524569e-02,4.54310536e-01],[-3.40264976e-01,7.31335998e-01,4.30978350e-02]],[[7.77367782e-03,-3.35110389e-02,-1.93999276e-01],[-6.54109776e-01,-4.33912247e-01,-1.44203675e+00],[-3.02088857e-01,-1.01751812e-01,1.32290006e-01]],[[3.61042917e-01,-4.99151558e-01,-1.09223890e+00],[4.34118629e-01,-2.38849655e-01,4.51436281e-01],[-1.27356201e-01,-1.90025687e-01,2.12270752e-01]],[[-1.67009253e-02,-7.04923809e-01,7.30735898e-01],[1.94246620e-01,3.05148423e-01,5.85667610e-01],[1.30122960e-01,2.25082755e-01,7.39642859e-01]],[[-4.88933980e-01,4.86959994e-01,5.84469020e-01],[1.48596808e-01,2.99986396e-02,3.75656694e-01],[-9.51979458e-02,-4.49403912e-01,4.68949378e-01]],[[-7.60037422e-01,5.47125697e-01,6.86295331e-02],[-5.06450713e-01,-3.82074952e-01,-2.88354419e-02],[4.59024280e-01,-4.55245003e-02,-9.00618210e-02]],[[5.39116561e-01,6.65699720e-01,4.87886339e-01],[2.14969311e-02,5.61352372e-01,-6.17261752e-02],[2.15786964e-01,-1.84001192e-01,-5.70566356e-01]],[[-3.10673177e-01,-6.31435886e-02,5.06996572e-01],[-5.69995761e-01,1.31119922e-01,4.01697576e-01],[2.42410839e-01,-5.61803281e-01,-1.75878834e-02]],[[1.81285381e-01,-3.26784939e-01,2.60848820e-01],[-2.00914279e-01,-2.27246042e-02,1.36462167e-01],[-6.01935089e-01,3.83028053e-02,2.32882142e-01]],[[-1.22746654e-01,7.06771433e-01,-8.68706763e-01],[7.11097643e-02,-2.69179642e-01,-5.44108033e-01],[-8.49791229e-01,1.08850673e-01,6.59972310e-01]],[[-7.20804036e-02,-2.73552775e-01,-2.81542033e-01],[6.47670925e-01,-6.84743235e-03,2.68869549e-01],[1.14327334e-01,-3.25942010e-01,-7.12384641e-01]],[[5.67118704e-01,-1.80826411e-01,-6.72466516e-01],[5.55270584e-03,3.72706413e-01,3.72635037e-01],[-3.98413092e-01,6.69964731e-01,1.11944348e-01]],[[6.85095429e-01,-4.42386121e-01,2.26707205e-01],[-5.01169622e-01,-4.20224875e-01,2.37142891e-01],[-8.71770680e-01,4.00556326e-01,6.54482663e-01]],[[-7.54614711e-01,-4.71185781e-02,2.49034688e-01],[-3.58341217e-01,-1.63783133e-01,-9.04484391e-02],[-5.04411519e-01,2.95977384e-01,-1.60237670e-01]],[[-4.25534576e-01,4.71477419e-01,-2.85018273e-02],[-3.28787208e-01,6.31451309e-01,-4.21287790e-02],[-1.23763096e+00,8.50815535e-01,3.84354830e-01]],[[-3.85776371e-01,7.93015137e-02,-8.97404134e-01],[5.07899106e-01,1.16467297e-01,-8.35398972e-01],[8.19352806e-01,-1.03359064e-02,2.99448252e-01]],[[-6.59045205e-03,-5.77046387e-02,1.40225347e-02],[-7.94860534e-03,5.65246105e-01,-5.90015829e-01],[5.35621226e-01,4.86009777e-01,-6.25289306e-02]],[[5.81517100e-01,-1.34199679e-01,-2.17293650e-01],[4.13571239e-01,4.66539323e-01,2.19019040e-01],[7.27247119e-01,4.34516758e-01,3.30286443e-01]],[[-3.41348380e-01,6.72028363e-02,-4.01834808e-02],[-2.54339371e-02,-1.63620561e-01,-8.65067169e-02],[-2.25622267e-01,6.85440123e-01,-2.10286617e-01]],[[9.17414606e-01,-2.69205928e-01,-2.21105397e-01],[-1.48783833e-01,9.37692523e-02,5.88051856e-01],[8.39023888e-01,-2.18918487e-01,-4.14471716e-01]],[[2.98305433e-02,-2.42523268e-01,-2.12582543e-01],[-4.90942508e-01,7.31087565e-01,-4.81498539e-01],[-6.83077216e-01,-2.10226014e-01,-1.39056504e+00]]],[[[1.16452646e+00,-1.14265561e+00,-1.64932668e-01],[-1.38318032e-01,3.65344197e-01,6.07423902e-01],[-2.08340585e-01,-3.35514754e-01,4.40570623e-01]],[[-1.33514571e+00,1.12712252e+00,-7.41254330e-01],[-1.82252839e-01,-4.71825868e-01,-7.78435528e-01],[-1.74036950e-01,3.57532740e-01,-7.10217774e-01]],[[2.20250607e-01,-3.30652893e-02,2.55587757e-01],[-3.39011282e-01,1.04841627e-01,6.11229017e-02],[-4.19462025e-01,9.62606907e-01,2.97463443e-02]],[[1.08285166e-01,7.27905750e-01,-1.13310361e+00],[-9.54382196e-02,1.80913806e-01,2.97874898e-01],[6.70865715e-01,-1.83863834e-01,-1.13883331e-01]],[[-2.64182210e-01,2.99430221e-01,2.54610240e-01],[-3.46497089e-01,-1.70065627e-01,-1.16218761e-01],[-1.51430905e-01,3.77642095e-01,-7.65321255e-01]],[[-1.69198245e-01,-1.25449374e-01,5.96818365e-02],[-3.53426814e-01,1.19284891e-01,1.52143613e-01],[5.12975872e-01,4.78216022e-01,-8.69641006e-02]],[[7.74240270e-02,-2.41435096e-01,-3.66378397e-01],[5.25946558e-01,-6.98024631e-01,-5.94158471e-01],[-3.72160733e-01,4.05211389e-01,5.21005929e-01]],[[-7.34172523e-01,1.47274554e-01,1.18023150e-01],[-7.91205287e-01,4.57433552e-01,4.11453024e-02],[-1.25833437e-01,-1.11782411e-02,-6.24559462e-01]],[[-2.62340963e-01,1.68028370e-01,-8.35651875e-01],[-4.11995947e-01,-9.46577787e-02,5.76703906e-01],[-1.57304898e-01,-2.84119427e-01,-3.57242078e-01]],[[4.38606329e-02,-1.31793275e-01,-1.82894841e-01],[-6.30527139e-01,3.28649789e-01,2.11808905e-01],[-3.35360467e-01,-4.54902291e-01,5.32297134e-01]],[[4.30999905e-01,1.04880460e-01,1.10040434e-01],[-1.72114193e-01,7.60869622e-01,-1.84858158e-01],[1.34432539e-01,2.90112019e-01,2.00527124e-02]],[[4.57372576e-01,2.27905754e-02,2.65571773e-01],[-4.94554937e-01,-2.48126373e-01,1.48047879e-01],[5.05649090e-01,1.04156047e-01,5.85720390e-02]],[[-2.59385109e-01,-4.01955903e-01,-1.53772831e-01],[-6.90136135e-01,4.28696752e-01,2.57303238e-01],[2.22862810e-01,9.58140567e-02,3.36631328e-01]],[[8.19196701e-02,-4.12637800e-01,3.30888510e-01],[-9.89536867e-02,-2.63158590e-01,-3.59700084e-01],[2.36004949e-01,-4.40293163e-01,-3.25638026e-01]],[[5.65787144e-02,-2.83951402e-01,-1.04688942e-01],[1.47957057e-01,-5.98475756e-03,-1.22837797e-01],[7.15838790e-01,-5.50971806e-01,-3.44556510e-01]],[[-1.43631905e-01,3.35109174e-01,2.26978153e-01],[-9.38155353e-02,-2.14946896e-01,-3.47833373e-02],[-8.52356672e-01,1.16487432e+00,-3.11461657e-01]],[[-3.39324921e-01,-2.77666241e-01,1.69729203e-01],[-2.40737736e-01,-1.23629354e-01,-6.60977244e-01],[-8.32592189e-01,3.22019339e-01,-5.70386887e-01]],[[-2.39816621e-01,4.92802054e-01,5.69599867e-01],[7.19344020e-02,-1.46762669e-01,3.50944728e-01],[-6.77471638e-01,-3.38315099e-01,-2.64703393e-01]],[[-3.98946613e-01,-6.57643855e-01,-1.00997913e+00],[2.10681602e-01,-8.10534060e-02,-1.92705065e-01],[-3.15988213e-01,3.59894365e-01,-2.67255176e-02]],[[1.76271781e-01,2.99679875e-01,-2.51734376e-01],[-2.09557354e-01,2.57112741e-01,2.53417760e-01],[-2.63553351e-01,-3.77476841e-01,1.16930294e+00]],[[-2.57758200e-01,-1.18915997e-01,-2.12106466e-01],[-1.00550838e-01,-8.54169369e-01,-6.92118108e-02],[-3.47132236e-01,2.61482507e-01,4.22904998e-01]],[[7.81625330e-01,8.14743042e-02,-8.99016976e-01],[8.88864517e-01,-3.49422451e-04,-3.51271331e-01],[2.68901557e-01,2.77662694e-01,3.59414130e-01]],[[1.97846621e-01,7.12587953e-01,-2.47748971e-01],[-1.25888571e-01,-3.79664868e-01,-6.38128281e-01],[1.65616143e-02,8.87026846e-01,4.78557110e-01]],[[4.91684722e-03,-3.58152062e-01,1.57007843e-01],[-2.99445331e-01,-4.08542603e-01,5.04168689e-01],[-3.90955389e-01,-1.47962600e-01,2.64510721e-01]],[[-3.57123017e-01,-4.36279058e-01,-2.86097616e-01],[2.68792212e-01,5.09400487e-01,-9.87979248e-02],[-5.03688166e-03,-1.19462490e-01,2.41532579e-01]],[[4.37249154e-01,1.66423455e-01,6.21384919e-01],[3.58356744e-01,3.35431010e-01,5.94024181e-01],[-4.15161520e-01,-1.07536957e-01,-8.07327688e-01]],[[1.41281769e-01,-3.55208844e-01,2.02050693e-02],[-1.18178710e-01,-6.79052919e-02,-1.66814983e-01],[2.75945663e-02,5.88441379e-02,4.40765351e-01]],[[-2.03496590e-01,3.29023391e-01,-4.99566048e-01],[1.15207732e-01,-5.74654639e-02,-1.78311050e-01],[-2.53634036e-01,-7.89748356e-02,6.01288319e-01]],[[-2.18827844e-01,1.80634961e-01,3.00709665e-01],[-2.29798928e-01,-4.57121700e-01,2.64496475e-01],[2.98921704e-01,2.88208015e-02,-6.66399777e-01]],[[1.47815868e-01,2.00261455e-03,1.48916736e-01],[8.50606337e-02,1.53404400e-01,3.30797434e-01],[-2.06517503e-01,-3.30435112e-02,-6.36154532e-01]],[[-5.73891521e-01,-9.79450792e-02,1.45565495e-01],[-4.10294652e-01,1.29546642e-01,-4.80386853e-01],[-1.65568277e-01,9.98970270e-02,-1.19869500e-01]],[[-2.46191859e-01,-3.36972237e-01,-1.56358212e-01],[6.88185215e-01,1.28926814e-01,-9.08578396e-01],[5.32797337e-01,-1.17620945e+00,7.02770829e-01]],[[-1.05911553e-01,5.30425847e-01,-2.98424959e-01],[6.31502047e-02,3.23419832e-02,6.58756196e-01],[-5.84988654e-01,-5.27879238e-01,-7.02149644e-02]],[[-8.65609407e-01,-1.21415012e-01,2.49444559e-01],[-8.54371965e-01,7.95497775e-01,3.68765235e-01],[-6.83953404e-01,-4.29063559e-01,-1.23385811e+00]],[[-7.50041241e-03,1.39312774e-01,-1.78295560e-02],[2.69168615e-02,1.71477512e-01,-4.63843584e-01],[-9.55600202e-01,5.92180431e-01,8.59549046e-02]],[[-3.23579460e-01,6.27334714e-01,-2.68392026e-01],[2.50029057e-01,4.50786084e-01,6.07930541e-01],[-4.30082381e-01,1.76416904e-01,-2.21151039e-01]],[[9.30623114e-01,9.79368627e-01,2.59132922e-01],[-1.08139608e-02,-5.66332527e-02,1.23414986e-01],[7.47662067e-01,-4.26898479e-01,-8.61300409e-01]],[[5.09499013e-01,-3.19017082e-01,2.13202164e-01],[1.87600851e-01,-8.68662179e-01,3.92770916e-01],[-2.50089675e-01,-1.07995667e-01,1.85431376e-01]],[[6.44268751e-01,-2.39125416e-01,4.28586781e-01],[3.09539109e-01,-6.13754869e-01,-5.43072820e-01],[5.72737515e-01,3.56176138e-01,-3.57994854e-01]],[[-2.10838184e-01,9.12682563e-02,-2.11475223e-01],[-7.24975705e-01,-2.50210285e-01,3.29584628e-01],[-8.62017751e-01,5.20287871e-01,-9.05730247e-01]]],[[[-1.13821185e+00,3.34901750e-01,-4.51849848e-01],[3.75644952e-01,3.89098912e-01,-6.91162109e-01],[-6.71545148e-01,4.55752134e-01,1.43017799e-01]],[[1.00684476e+00,1.15836062e-01,7.09279418e-01],[4.09633458e-01,-5.55170953e-01,2.76301503e-01],[5.92172980e-01,5.86933307e-02,2.40859285e-01]],[[1.43211007e-01,-6.18210077e-01,-1.36749327e-01],[7.54583299e-01,3.36431652e-01,-1.83357552e-01],[-1.85787097e-01,2.23608822e-01,4.69089538e-01]],[[-5.76510072e-01,6.58178106e-02,3.25739443e-01],[1.86184242e-01,7.99657166e-01,6.53242320e-02],[1.28007501e-01,8.72514367e-01,3.56106199e-02]],[[-1.08828509e+00,-3.74411017e-01,2.51079708e-01],[5.62957704e-01,4.53837663e-01,-2.54458696e-01],[-2.20393866e-01,-9.57194149e-01,-3.85980368e-01]],[[-2.74972379e-01,-6.97813451e-01,-2.72497326e-01],[1.36187196e-01,8.19003880e-01,-2.06688687e-01],[-9.67464894e-02,7.92453051e-01,3.89437340e-02]],[[4.15201485e-01,-3.62185925e-01,-7.29359910e-02],[-4.37333673e-01,9.10096318e-02,2.47977912e-01],[-4.59291786e-01,6.49008334e-01,3.02289367e-01]],[[1.27785176e-01,-1.27376750e-01,6.35046780e-01],[7.87282050e-01,3.25838365e-02,-3.70437689e-02],[8.10589373e-01,3.97883564e-01,-4.28905308e-01]],[[-7.35499203e-01,-1.58862799e-01,4.83877122e-01],[-6.51264608e-01,3.30604404e-01,-7.34452680e-02],[-1.54429212e-01,-3.79199415e-01,-1.91543140e-02]],[[3.17239434e-01,2.55642850e-02,-2.62093961e-01],[6.04179561e-01,-1.74598731e-02,-2.44667768e-01],[1.20656657e+00,-6.33288682e-01,6.34272456e-01]],[[-2.33174056e-01,2.07386330e-01,3.20911944e-01],[4.32841033e-02,-1.56337786e-02,-1.98392078e-01],[8.70857164e-02,-3.90643924e-01,-8.82484987e-02]],[[-9.58839245e-03,-4.67887372e-01,-3.11059237e-01],[3.65432650e-01,-1.53650969e-01,4.51006919e-01],[7.61927605e-01,2.94866592e-01,5.99392116e-01]],[[3.00283432e-01,2.98919350e-01,5.19796848e-01],[-7.16036141e-01,4.76788700e-01,1.42822310e-01],[-3.77395213e-01,1.33670092e-01,5.96323371e-01]],[[1.01777412e-01,-9.89999920e-02,-1.02492385e-01],[9.37378779e-02,-2.75622040e-01,1.12406999e-01],[-5.92367053e-01,1.19905524e-01,-4.70834881e-01]],[[-5.13792753e-01,-5.84535241e-01,3.95982563e-01],[1.26492366e-01,-2.87825078e-01,-4.80481863e-01],[-5.84805310e-01,3.08595389e-01,-2.11832076e-01]],[[5.55290319e-02,2.57198840e-01,-1.85072958e-01],[4.38944429e-01,1.77438363e-01,-3.50215048e-01],[-5.59045255e-01,5.65775454e-01,4.13373321e-01]],[[1.01731956e-01,-1.67199761e-01,1.48256645e-01],[3.00477445e-01,2.17009475e-03,5.08205175e-01],[7.00857937e-01,-1.01262845e-01,-7.05159962e-01]],[[-7.23469734e-01,3.15559924e-01,4.90404554e-02],[4.20326054e-01,-2.49855757e-01,6.75835311e-02],[-4.43178453e-02,9.50075626e-01,1.12397835e-01]],[[5.14363647e-01,-2.30295688e-01,-2.73886859e-01],[7.23618090e-01,-1.28154263e-01,2.49530211e-01],[-7.67974436e-01,9.80997533e-02,3.45214754e-01]],[[-7.53857136e-01,3.81794989e-01,7.26363286e-02],[1.35016933e-01,1.01340339e-01,2.40874231e-01],[-8.77865329e-02,1.23512156e-01,1.19145358e+00]],[[3.88328880e-01,8.13169539e-01,-1.00516582e+00],[1.97826117e-01,-4.31959063e-01,-4.46781695e-01],[-9.20853913e-02,-2.63966233e-01,7.42676184e-02]],[[-1.35597929e-01,7.41456032e-01,-3.02851081e-01],[5.16342700e-01,-2.91418303e-02,2.84397081e-02],[-9.85383056e-03,-8.51814747e-02,2.56074280e-01]],[[5.72882295e-01,4.85656977e-01,1.17570519e+00],[-2.20934525e-01,-4.58400756e-01,5.49551547e-01],[-4.60978039e-02,2.79981464e-01,4.45908040e-01]],[[-3.97208176e-04,-3.04332018e-01,1.42749563e-01],[-8.52214098e-01,1.06154270e-01,4.04840380e-01],[-1.58882767e-01,4.77999747e-01,-9.82423276e-02]],[[7.62067378e-01,2.61584044e-01,5.15913188e-01],[5.58919370e-01,3.56966406e-01,7.18562841e-01],[1.28622150e+00,5.68438647e-03,7.08205044e-01]],[[1.84576154e-01,-1.29373342e-01,-5.39798021e-01],[3.63481045e-01,-1.50102556e-01,3.47441696e-02],[-4.38762039e-01,6.81540251e-01,7.00728476e-01]],[[-5.08494973e-01,5.01077950e-01,5.31041212e-02],[-5.39526343e-01,5.97962737e-01,5.49443901e-01],[-3.15254450e-01,2.22018987e-01,6.35778010e-01]],[[-1.12664096e-01,4.27426815e-01,3.54415834e-01],[3.37698400e-01,2.86288053e-01,9.21589434e-01],[-4.36071277e-01,-2.91267838e-02,8.01375329e-01]],[[-4.68276024e-01,4.84275132e-01,-8.92139256e-01],[2.09140718e-01,-1.56354681e-01,-4.66821253e-01],[-7.84538686e-01,8.62649903e-02,-3.92677426e-01]],[[-4.68592286e-01,1.16899632e-01,4.38994132e-02],[2.75111180e-02,-2.24273130e-01,-2.82501966e-01],[6.04770720e-01,-2.86246955e-01,-1.12491988e-01]],[[5.45976400e-01,-7.48801887e-01,4.42359507e-01],[2.43474633e-01,-7.84554034e-02,4.41319466e-01],[9.94270027e-01,-1.99255660e-01,-6.49481080e-04]],[[8.98720324e-02,-6.58144414e-01,5.13095021e-01],[4.52568650e-01,-8.04218709e-01,-8.35053623e-01],[-3.16255912e-02,3.60518135e-03,7.07347155e-01]],[[2.13459015e-01,2.28609219e-01,1.04247563e-01],[-6.41995575e-03,1.02398666e-02,2.36153141e-01],[6.65452659e-01,1.02055632e-01,7.26885855e-01]],[[-1.29608572e-01,-7.82554090e-01,3.34359974e-01],[5.09475172e-01,-5.56932151e-01,1.52789280e-01],[-3.05722773e-01,-1.89243909e-02,1.62964717e-01]],[[-9.65975642e-01,-3.01639140e-01,7.18363225e-01],[-1.52490765e-01,-8.58663321e-01,1.92619950e-01],[-3.30246031e-01,-3.49354237e-01,1.33983463e-01]],[[3.06246758e-01,-2.23469645e-01,2.87193239e-01],[-1.89356223e-01,-7.56302834e-01,4.11482118e-02],[3.77195239e-01,-6.28044903e-01,4.72502321e-01]],[[-5.18002570e-01,-3.19778532e-01,3.53713900e-01],[3.27177376e-01,-3.97561014e-01,-2.54354719e-02],[-8.15919876e-01,-2.26801842e-01,4.83231902e-01]],[[-1.29070887e-02,5.67190111e-01,1.40007615e-01],[2.38275677e-01,-2.91855987e-02,2.25397810e-01],[3.71582270e-01,1.49177939e-01,-1.39236832e-02]],[[-7.74696529e-01,-5.80773413e-01,6.43971637e-02],[-2.94497699e-01,6.00312889e-01,-2.58231312e-01],[-7.92041361e-01,-3.98598909e-02,5.81820965e-01]],[[3.67043242e-02,-5.54631889e-01,4.58565086e-01],[9.30221602e-02,-4.58336473e-01,5.38203597e-01],[2.13214651e-01,-1.40118375e-01,-1.53735071e-01]]],[[[4.77788866e-01,-2.19630599e-01,8.30252171e-01],[3.48405391e-01,5.36315203e-01,3.72101933e-01],[3.08978260e-01,-5.39346457e-01,2.32924178e-01]],[[-7.51258060e-02,-6.30404949e-01,-3.84038389e-01],[1.16882764e-01,-3.22004467e-01,1.06867708e-01],[3.67042333e-01,-1.75034881e-01,6.60319207e-03]],[[2.76163578e-01,8.02230835e-02,-2.40594015e-01],[-7.83054275e-04,2.04929277e-01,-1.71694711e-01],[-1.56729758e-01,6.94492459e-02,6.28135502e-02]],[[2.96653248e-02,1.09709371e-02,-5.24143159e-01],[-3.08657885e-01,-2.90899754e-01,-2.72286892e-01],[8.22048128e-01,6.29773503e-03,1.03097975e-01]],[[2.84571052e-01,7.77390227e-02,4.82588798e-01],[-1.68324694e-01,-6.11665938e-03,2.55018562e-01],[1.05191305e-01,7.82308728e-02,9.25568342e-01]],[[-4.32708323e-01,-6.37667775e-02,-7.49093667e-02],[-2.93617725e-01,-7.07713515e-02,-9.13780779e-02],[-3.14429045e-01,-2.14572698e-01,-5.28202891e-01]],[[-2.07435042e-01,1.93124384e-01,-3.13356459e-01],[3.04211795e-01,3.98812145e-02,-5.79248294e-02],[4.36798334e-01,-4.83083501e-02,-5.19423420e-03]],[[-4.05780286e-01,4.41058099e-01,-4.15295660e-01],[3.81441504e-01,2.82094050e-02,3.24103862e-01],[-2.67370760e-01,4.99000013e-01,1.76501364e-01]],[[-3.93368483e-01,-4.40481067e-01,-8.18545640e-01],[3.80854979e-02,-5.87116897e-01,-6.06510639e-01],[-1.00482523e+00,-1.35302812e-01,-7.60456085e-01]],[[-6.69401228e-01,-3.31538439e-01,-4.06218767e-01],[-6.14137292e-01,3.74704123e-01,-1.04226835e-01],[-1.05029917e+00,-5.28568983e-01,3.84704679e-01]],[[3.96880805e-01,5.07856607e-01,-1.26707330e-01],[2.21882090e-01,-4.98957247e-01,4.25003707e-01],[-6.47019222e-02,5.32539725e-01,1.02745795e+00]],[[-2.31012747e-01,3.49951416e-01,2.15290129e-01],[1.56127755e-02,-3.68078016e-02,-4.78053570e-01],[-2.16560781e-01,1.76927730e-01,2.15460390e-01]],[[7.30324924e-01,3.77450973e-01,3.38300973e-01],[5.81148863e-01,-4.30706263e-01,2.51972489e-02],[1.42543644e-01,-1.82639629e-01,-1.28167763e-01]],[[-2.98127141e-02,-5.77139080e-01,-1.17802851e-01],[5.89525342e-01,-4.59614098e-01,1.97794765e-01],[1.66451836e+00,-1.60434425e-01,7.19423413e-01]],[[4.12349552e-01,-1.05099833e+00,5.30932605e-01],[7.52010524e-01,-4.60053205e-01,-1.02843963e-01],[1.37906647e+00,-2.35703424e-01,-2.03830358e-02]],[[6.94184527e-02,-3.21130804e-03,-1.23236403e-02],[-6.57596141e-02,-3.32210541e-01,3.98561180e-01],[2.77386189e-01,-8.07753354e-02,8.54701459e-01]],[[-5.25375783e-01,-1.76106170e-01,-6.37119263e-02],[-7.81808496e-01,4.52454835e-02,-3.99290383e-01],[-4.67956007e-01,-5.93528561e-02,1.98626090e-02]],[[3.97369385e-01,-3.26970071e-01,5.19788206e-01],[6.75440013e-01,-3.33265185e-01,9.50295031e-01],[5.39422452e-01,-4.32077572e-02,9.18825686e-01]],[[-3.69720280e-01,-4.57540333e-01,-1.23800182e+00],[3.09717745e-01,9.23740044e-02,-3.35929506e-02],[3.45318131e-02,4.55284044e-02,4.73785773e-02]],[[-1.60056621e-01,1.74146369e-01,-4.21466976e-01],[-3.84157188e-02,-3.11846763e-01,-2.22651929e-01],[-9.19226170e-01,-5.16447425e-01,-1.14739573e+00]],[[1.50118083e-01,5.27484834e-01,8.60395491e-01],[6.60678864e-01,2.55311783e-02,4.11196828e-01],[2.69454181e-01,-2.46371124e-02,8.50913703e-01]],[[-1.42896819e+00,-5.17264068e-01,-7.01924562e-01],[-3.53174537e-01,-3.90093476e-01,6.50093108e-02],[-6.50545299e-01,-2.68826097e-01,-3.10382813e-01]],[[-5.13570726e-01,-4.38462973e-01,-1.71866524e+00],[-2.12095767e-01,-2.95868695e-01,-1.03604269e+00],[3.72187614e-01,-3.64867151e-01,-1.18372858e-01]],[[1.84257239e-01,-5.18949986e-01,6.12291217e-01],[-3.93599987e-01,1.79458976e-01,-1.30206555e-01],[1.94060579e-01,-3.88399465e-03,-9.56223756e-02]],[[-3.43708068e-01,-3.95309478e-01,-5.60418367e-01],[-5.97912312e-01,1.49966208e-02,-2.28786364e-01],[-9.25872684e-01,-2.37139136e-01,-1.13412738e+00]],[[-1.66263387e-01,-3.35333139e-01,-3.34688216e-01],[5.68370447e-02,2.07914442e-01,6.05946183e-01],[6.07987821e-01,1.84205681e-01,1.39000505e-01]],[[3.42064559e-01,9.54314023e-02,-2.59534776e-01],[7.24494979e-02,-2.10096110e-02,-5.77441514e-01],[-1.74948841e-01,-5.16038276e-02,-5.63312352e-01]],[[-4.93575275e-01,3.15334082e-01,3.84654820e-01],[-4.70685989e-01,1.46469831e-01,2.24204987e-01],[-2.06070438e-01,-6.58427417e-01,6.93950653e-02]],[[4.44783330e-01,6.73437953e-01,1.37004423e+00],[-2.95151528e-02,1.22663158e-03,7.49979794e-01],[-5.86828530e-01,-3.18083137e-01,1.73371866e-01]],[[3.51254374e-01,1.26614869e-01,-9.53669772e-02],[4.64570224e-01,-4.36764479e-01,8.33209336e-01],[8.41038883e-01,-2.19550401e-01,1.77319661e-01]],[[-2.14097247e-01,-3.33079517e-01,-1.15537353e-01],[3.47237964e-03,3.26226920e-01,1.49778174e-02],[-8.98198485e-02,3.66274595e-01,-2.29420170e-01]],[[1.03550471e-01,-9.14491713e-02,-3.07320327e-01],[-3.93527895e-01,5.37676454e-01,-3.55886728e-01],[3.11889887e-01,4.05579239e-01,-7.81489134e-01]],[[-7.26342261e-01,-2.04968333e-01,-8.29914033e-01],[-1.28618315e-01,-2.10944191e-01,-5.54079488e-02],[-5.56467593e-01,-3.43824863e-01,-2.90058106e-01]],[[-1.77682653e-01,-1.11021465e-02,-1.15509599e-01],[-6.82530925e-02,-1.20998330e-01,1.31390318e-01],[-1.50633410e-01,4.54015970e-01,3.89263704e-02]],[[4.67229545e-01,-1.29281685e-01,-3.47042799e-01],[4.63922292e-01,2.01871052e-01,1.85917363e-01],[1.76037282e-01,-1.38452768e-01,1.02118301e+00]],[[-3.22294235e-01,-4.10035491e-01,-6.38107955e-01],[3.87926608e-01,-7.40915760e-02,4.00345832e-01],[-5.66465080e-01,-3.15416723e-01,-2.16310918e-01]],[[6.03785813e-01,3.29935700e-01,1.27253858e-02],[2.73108244e-01,6.98594630e-01,-5.60960412e-01],[5.61771929e-01,-1.43722579e-01,6.13770902e-01]],[[-2.70237118e-01,-2.48578131e-01,1.03181703e-02],[-4.11390126e-01,-1.76772565e-01,-8.11704397e-02],[-8.34773421e-01,-5.10621257e-02,2.27374196e-01]],[[1.10822701e+00,2.66319156e-01,9.61411893e-01],[-1.03237800e-01,1.42710000e-01,2.60163993e-01],[-8.21252689e-02,3.83240223e-01,-1.88177064e-01]],[[-3.15191627e-01,4.89187807e-01,-2.80365646e-01],[-2.82604724e-01,-9.58936065e-02,1.46079049e-01],[1.62140623e-01,-1.95797738e-02,8.41009736e-01]]],[[[3.39444578e-01,1.51678756e-01,8.16414297e-01],[-3.69029343e-02,-1.11067399e-01,-2.44874030e-01],[3.49200249e-01,-1.11889848e-02,7.31045827e-02]],[[-5.82361698e-01,-4.67205673e-01,2.84433603e-01],[-4.56725061e-01,-7.30037689e-01,-1.05049178e-01],[-2.29470834e-01,7.40045726e-01,5.46926737e-01]],[[4.69546914e-01,1.44374716e+00,-3.31481552e+00],[-5.97461201e-02,1.68686941e-01,9.54759002e-01],[2.73440510e-01,6.75578237e-01,-8.25126708e-01]],[[7.76623413e-02,4.15533066e-01,-5.92425525e-01],[-5.13824560e-02,-2.74954855e-01,-2.98534390e-02],[-8.87590349e-02,3.65583599e-02,-5.32692254e-01]],[[3.19398254e-01,-1.75248787e-01,4.01119977e-01],[2.68542945e-01,4.80162323e-01,1.13028415e-01],[-2.52586246e-01,-4.57555681e-01,-2.78715104e-01]],[[1.06152199e-01,8.06702822e-02,2.44733766e-01],[-2.22174808e-01,-5.39705716e-02,2.67648071e-01],[5.34193367e-02,-3.37182581e-02,1.02967009e-01]],[[1.46328971e-01,9.82381403e-02,-1.12747121e+00],[-5.54404318e-01,-3.43465865e-01,-7.13659525e-01],[-6.60491586e-02,7.23045766e-02,1.19273320e-01]],[[5.26544750e-01,-1.18742622e-01,-3.71568471e-01],[2.19802737e-01,-4.14533585e-01,8.07047486e-01],[2.78853595e-01,-4.25003767e-01,6.54214174e-02]],[[1.61160767e-01,9.45518836e-02,1.03997871e-01],[-1.47512734e-01,-2.10505784e-01,9.31074679e-01],[3.37849885e-01,4.19580787e-01,1.02147901e+00]],[[-1.80997357e-01,-4.02051270e-01,3.27338874e-01],[-1.02980733e+00,2.78344274e-01,-7.18467891e-01],[-6.52851686e-02,7.27208793e-01,-7.39239991e-01]],[[-1.92534164e-01,-1.05627549e+00,7.60547161e-01],[6.45733118e-01,3.44769597e-01,-6.87719643e-01],[-3.10850054e-01,-6.22068644e-01,6.36556223e-02]],[[-6.41264439e-01,-3.72444719e-01,-4.85879421e-01],[-3.09348223e-04,5.65199219e-02,-3.13606769e-01],[-8.93916190e-01,3.25655162e-01,-1.48341835e-01]],[[3.94782349e-02,-3.23757291e-01,8.14656019e-01],[-2.94313997e-01,2.81141043e-01,-1.16761528e-01],[5.10774314e-01,-1.28152907e-01,-2.25144267e-01]],[[-1.82522237e-01,-1.60685889e-02,-4.64905441e-01],[2.90639270e-02,-3.94562423e-01,2.41924807e-01],[5.51729202e-02,4.40139383e-01,-4.83533919e-01]],[[1.55664653e-01,-1.84993830e-03,5.23815632e-01],[7.09892213e-01,-2.44468093e-01,7.11236373e-02],[1.79701105e-01,1.08549707e-01,5.62486410e-01]],[[4.34011459e-01,-7.47964680e-01,-4.59858596e-01],[5.09448528e-01,-8.16427827e-01,6.50094271e-01],[2.01898381e-01,1.26199558e-01,-2.83016972e-02]],[[2.05627099e-01,-7.73985237e-02,-4.21843469e-01],[-1.41622722e-01,1.63785666e-01,-3.70392323e-01],[-3.22820246e-01,2.32061625e-01,-3.88904326e-02]],[[-4.42088693e-01,8.06450173e-02,-2.04405412e-02],[2.38806918e-01,-1.20905900e+00,-5.15304506e-03],[4.14281487e-01,5.39619386e-01,4.83835995e-01]],[[-6.13918483e-01,-1.14479810e-01,-4.40492719e-01],[-7.88843036e-02,-2.43925616e-01,-6.52662218e-01],[2.32323080e-01,4.32538450e-01,-2.94523776e-01]],[[-3.86299521e-01,-5.51304340e-01,2.95230240e-01],[2.04539560e-02,2.70600349e-01,1.74702574e-02],[1.78343579e-01,-1.29349366e-01,1.08822182e-01]],[[4.30599451e-01,4.19245690e-01,-3.20055306e-01],[-3.47112119e-01,2.26538599e-01,3.89617473e-01],[-1.82788268e-01,-6.20426759e-02,1.68309927e-01]],[[2.57614609e-02,3.34575862e-01,2.96914876e-01],[7.82759711e-02,-2.75460631e-01,-8.11714888e-01],[-1.34502366e-01,-6.37992561e-01,3.62206370e-01]],[[5.97840905e-01,-4.34888154e-01,-3.68740916e-01],[-1.09654620e-01,2.62232035e-01,-1.39001831e-01],[-5.24420142e-02,1.28144741e-01,-3.19537908e-01]],[[1.56658411e-01,1.39415458e-01,3.82094264e-01],[9.44329575e-02,-5.76974094e-01,-4.37697582e-02],[1.17238872e-01,-1.84615299e-01,7.50725925e-01]],[[4.05776463e-02,-5.85504413e-01,4.73171264e-01],[3.61992687e-01,-2.35180929e-01,-6.61306798e-01],[-2.74122089e-01,-2.31290638e-01,9.56963524e-02]],[[7.97097459e-02,-9.38244089e-02,-5.69207184e-02],[-3.25706780e-01,2.72145450e-01,-5.09883642e-01],[1.02898443e+00,2.10009277e-01,2.77764887e-01]],[[-9.82356146e-02,-1.51537657e-01,8.63725662e-01],[-5.33411026e-01,3.74289066e-01,6.42766893e-01],[-3.94454263e-02,1.37075722e-01,1.18276393e+00]],[[-1.21826738e-01,-6.50967836e-01,3.71312350e-01],[-8.45313370e-02,-3.13133419e-01,-5.03309429e-01],[2.15497956e-01,-8.90124664e-02,-2.17835560e-01]],[[-4.26200926e-02,-4.50619549e-01,-3.67839634e-02],[8.07890072e-02,-1.75707310e-01,6.56189501e-01],[2.96463490e-01,-3.24407578e-01,-8.12443532e-03]],[[1.28448725e-01,2.10395567e-02,-7.20937729e-01],[4.80381310e-01,-3.60033393e-01,-1.21055230e-01],[4.08531249e-01,9.18144435e-02,4.38117981e-02]],[[8.84184659e-01,-5.17623663e-01,6.81385875e-01],[-2.60305226e-01,-2.41974294e-01,-2.16779903e-01],[2.12686494e-01,8.09948966e-02,6.64508402e-01]],[[-1.16281681e-01,1.11734383e-01,-1.39262342e+00],[-6.24999285e-01,4.60733742e-01,-9.58506107e-01],[5.17768323e-01,-8.70120600e-02,-1.71885502e+00]],[[-2.57189512e-01,3.24366055e-02,5.64755917e-01],[2.23578900e-01,1.88332021e-01,-2.52870798e-01],[6.01629317e-01,-5.24903648e-02,-8.05902660e-01]],[[-1.68021336e-01,6.04971588e-01,4.97301877e-01],[2.17706129e-01,3.88530850e-01,2.62524813e-01],[-4.84942161e-02,2.77968973e-01,3.31915051e-01]],[[-1.65161137e-02,-4.57514048e-01,8.84105086e-01],[7.12522745e-01,3.20286483e-01,-5.15619516e-01],[7.52765089e-02,5.63875020e-01,3.46151531e-01]],[[-2.05569878e-01,2.21664697e-01,5.46630323e-01],[-4.91531730e-01,5.76975644e-01,4.17969733e-01],[-2.07396135e-01,7.72177041e-01,-7.93689072e-01]],[[2.48510137e-01,8.08591306e-01,-1.08031952e+00],[-4.37262058e-01,1.03729509e-01,-3.92631799e-01],[-8.37760642e-02,4.03707087e-01,3.71094376e-01]],[[-1.69268399e-02,5.51252365e-01,-5.72839200e-01],[-4.66998547e-01,4.67495397e-02,9.51392055e-01],[-5.30746520e-01,6.75509796e-02,-1.59152910e-01]],[[2.52704829e-01,-9.75069106e-02,5.05124591e-02],[-1.99819580e-01,-4.48437780e-01,-5.91776967e-01],[-4.90773112e-01,-2.57554259e-02,-5.57421863e-01]],[[2.69498587e-01,6.98155835e-02,1.09765756e+00],[-2.10129842e-01,-5.21637797e-01,-1.36575341e-01],[2.97381580e-01,-3.86518866e-01,5.55005848e-01]]],[[[0.3548838,0.19759478,0.54238576],[0.90774584,0.03796746,0.2415039],[0.73393905,0.11108948,-0.26975307]],[[0.44717443,0.61241055,-0.0529421],[0.05997957,0.93160003,0.00420039],[-0.48184958,0.02527514,-0.21884383]],[[-0.18336397,0.28757578,-1.6374246],[-0.19113798,0.26857224,-0.21540369],[-0.03863283,-0.24235992,-0.70590156]],[[-0.10884998,0.5797504,0.01239788],[0.18410417,-0.35222638,0.09965169],[-0.3606109,0.18117252,-0.4734571]],[[-0.11898235,-0.04285732,0.5499092],[0.10602297,0.05233233,0.12062218],[0.28206718,-0.85800827,-0.27682406]],[[0.60508096,0.06456041,-0.934145],[-0.0539567,-0.03737906,0.5431613],[0.6481704,-0.06155114,-0.24136469]],[[0.11504144,0.38317594,0.16435288],[-0.03344981,0.4091569,0.14017256],[-0.06091351,-0.54584306,0.02962554]],[[-0.33904555,-0.95738333,0.59497666],[0.33073905,0.58497715,0.489336],[0.6460787,-0.32125717,-0.33690217]],[[0.35636452,0.26921254,-0.8125273],[0.23155944,0.37514102,-0.46112022],[-0.20050308,-0.13374013,-0.3877716]],[[-0.35261664,0.18491757,-0.04934869],[0.56227267,-0.37160194,-0.31814763],[0.13920517,-0.9248447,-0.6386826]],[[-0.00220434,0.0592375,-0.62890935],[0.15862152,0.6138665,-0.02052673],[-0.38747805,0.25663614,0.01411094]],[[-0.37721455,0.20312802,-1.1596438],[-0.00318099,0.59382385,-0.04460523],[0.37441415,0.24924403,-0.29413643]],[[0.1630857,0.02819036,0.3610039],[0.53379005,0.6534909,0.5967727],[0.22463395,-0.3467846,-0.3655774]],[[0.36068487,0.08452653,0.14018095],[-0.15738128,0.6154814,-0.28941244],[0.05049777,-0.6030346,-0.15033473]],[[0.34232295,0.47345394,0.3183271],[0.04088688,0.5577647,-0.40304306],[0.30246037,0.3198673,0.5458923]],[[-0.1786524,0.37777495,-0.04954645],[-0.64940816,-0.3452728,-0.33288985],[-0.5999491,-0.03961614,0.03671822]],[[0.22652882,0.62170017,-0.5504306],[0.46110412,0.30080974,-0.52533567],[0.05557792,0.34359875,-0.7343962]],[[0.4023102,-0.48979002,0.45859218],[0.5340653,-0.39891046,-1.1486527],[0.9309298,-0.44569105,-0.01288747]],[[-0.07887886,0.18581054,-0.35762283],[0.74445057,-0.15610886,-0.28214356],[0.6069288,-0.5257852,0.8033041]],[[-0.54343057,-0.25869238,0.79507345],[-0.3590848,-0.4001653,0.719189],[-0.50490564,-0.8630236,-0.8037561]],[[0.4943235,0.7584522,-0.23195587],[0.6472847,-0.1895538,-0.42006972],[0.04325182,-0.21203154,-0.2349158]],[[-0.7032177,0.14435352,0.82509077],[0.0185676,0.51036155,-0.6166643],[0.11610823,-0.1352677,0.5314345]],[[0.15190758,-0.3837674,0.17739636],[-0.00668654,0.04706678,1.0566692],[0.31933242,-0.18528609,0.47589323]],[[-0.5904284,-0.9733334,0.52601975],[0.25577635,-0.4375239,0.06671353],[0.13505018,-0.48138347,-0.64600325]],[[-0.08093917,0.05326762,0.32195795],[-0.5867742,-0.8207246,0.29632747],[-0.05172875,-0.9017748,0.18397984]],[[0.02865736,0.27508408,-1.256979],[-0.6739683,-0.21213834,0.08122447],[-0.14759202,0.10601176,-0.12251307]],[[0.03817049,0.42158014,0.8485191],[-0.02000443,-0.08184171,0.3212978],[-1.392438,-0.06972896,0.3776974]],[[0.25101677,0.6609719,0.5090004],[-0.15479872,0.4781894,-0.6169721],[-0.70148635,0.4371381,-0.8028824]],[[-0.276155,0.320207,0.05695039],[0.03538595,-0.06550545,-0.9030851],[0.24492119,0.48334283,1.0447018]],[[-0.10271742,-0.36127374,-0.46449444],[0.17454052,0.3519065,-0.31238183],[-0.8107271,-0.13243681,-0.23340006]],[[0.855668,0.43161604,0.5379037],[-0.5821443,-0.6910562,0.26887295],[-0.00317481,0.3128952,-0.36996895]],[[-0.3874364,0.10378716,-0.32137066],[-0.76072055,0.06841936,0.17581956],[-0.22566019,0.02167862,-0.76911753]],[[0.68481696,0.51766074,1.0896119],[0.00229039,0.6886601,-0.38121167],[0.6026898,-0.27795538,0.12405857]],[[0.52145886,0.04427421,0.70234174],[0.5020666,-0.0305047,-0.3392572],[0.03235517,0.38844353,1.0137235]],[[0.05416787,0.26344448,-0.5820101],[-0.33436126,-0.00347078,-0.07024204],[0.02293114,-0.5032547,-0.30538595]],[[-0.23193705,0.35458186,-0.18967253],[0.89420134,0.0607705,0.18892018],[0.38128334,-0.14326489,0.11594776]],[[0.06643417,0.17599635,-0.695182],[-0.10632441,0.4798198,-0.4746221],[0.71639156,0.03037844,1.1433076]],[[0.12505668,-0.68903613,-0.57621557],[-0.34510368,-0.3389873,-0.18036132],[-0.1863466,0.06369179,0.10869639]],[[-0.09298162,-0.5389799,0.12343243],[-0.2367194,-0.58357376,-0.43899813],[-0.01562396,-0.910741,-0.3348204]],[[0.8360863,-0.02675582,0.15605614],[-0.19888043,0.18844597,-0.45494258],[0.54950505,0.21719077,-0.3524036]]],[[[6.46478117e-01,-3.51922333e-01,1.29789606e-01],[-4.78723526e-01,-6.27933741e-01,-2.35186651e-01],[-2.00384986e-02,-5.88639796e-01,2.65968621e-01]],[[4.40286547e-02,2.74464369e-01,-2.07919270e-01],[-1.10112369e-01,4.79220837e-01,3.99660975e-01],[-3.61546099e-01,5.10910690e-01,-2.27590531e-01]],[[1.67040981e-03,5.88322282e-01,-1.19364029e-03],[5.48721254e-01,-1.52560426e-02,2.30655506e-01],[-3.72496933e-01,1.69031486e-01,-2.13009194e-01]],[[-7.54081964e-01,1.24171495e-01,3.06123108e-01],[2.90268719e-01,-3.89455050e-01,-1.02498882e-01],[-9.85051334e-01,1.78745359e-01,1.15180708e-01]],[[4.56326939e-02,7.51836225e-02,-1.84955642e-01],[1.75829887e-01,-3.13098617e-02,6.88973188e-01],[9.01266187e-03,-1.12847590e+00,1.53456936e-02]],[[4.97117311e-01,-6.66236579e-01,4.17274863e-01],[-3.60324323e-01,-2.89331228e-01,1.21736050e+00],[1.54760510e-01,2.93251276e-01,-6.99178576e-02]],[[1.88812256e-01,2.17828885e-01,3.54378700e-01],[7.31120929e-02,-3.76282811e-01,9.33663175e-03],[-6.24653757e-01,1.86944112e-01,6.37043715e-01]],[[-7.37975359e-01,1.08027317e-01,-4.92870539e-01],[-2.82334507e-01,-6.60861507e-02,-7.81176984e-02],[4.20167893e-01,-9.40020561e-01,7.94553578e-01]],[[-3.80416662e-01,2.97785163e-01,1.26181260e-01],[-3.59564453e-01,1.38761364e-02,1.96159229e-01],[-4.08248156e-01,-8.97508860e-01,6.83590531e-01]],[[7.28161037e-01,-5.39820135e-01,4.93675917e-01],[6.42914057e-01,2.89710462e-01,3.32021773e-01],[1.67296737e-01,-1.83437347e-01,-1.16968310e+00]],[[-3.93722922e-01,1.18253279e+00,-4.68095541e-01],[-1.02842999e+00,6.04377627e-01,3.19559216e-01],[4.29306746e-01,2.28700563e-01,2.12062616e-02]],[[-3.56422424e-01,6.91910088e-02,-4.04692233e-01],[-7.89170682e-01,-5.39611995e-01,-1.73217207e-01],[-1.41768068e-01,-2.91708112e-01,3.95349622e-01]],[[-1.33174524e-01,-2.63598770e-01,1.91360757e-01],[2.35617906e-01,-1.88202143e-01,6.37510538e-01],[-5.36401808e-01,-4.07925040e-01,-3.76650542e-01]],[[3.92090172e-01,-5.40589690e-01,4.12483305e-01],[-2.35023052e-01,2.90279657e-01,2.05410689e-01],[-5.53087890e-01,9.25635025e-02,2.64156349e-02]],[[-9.46347564e-02,3.51841934e-02,5.71878433e-01],[-4.52931464e-01,8.81099999e-02,3.16426396e-01],[1.04249403e-01,-3.27445447e-01,-6.45748258e-01]],[[4.46627289e-01,-4.76618201e-01,-3.94212812e-01],[9.02980566e-02,-3.55366111e-01,6.79093957e-01],[6.81436211e-02,4.03918833e-01,4.99248147e-01]],[[-2.15811387e-01,4.65349823e-01,2.69602209e-01],[-4.58452431e-03,-1.57302111e-01,1.34179279e-01],[5.96232533e-01,-6.28643751e-01,-3.49781215e-01]],[[-1.59131661e-01,-4.20489401e-01,6.47723675e-02],[-5.55822730e-01,3.44381183e-01,2.06458837e-01],[5.96683323e-01,-3.76540273e-01,-1.28278643e-01]],[[7.92503580e-02,-1.41477957e-01,3.47007483e-01],[4.42589313e-01,1.19212359e-01,-4.68866736e-01],[-1.26811516e+00,2.04383105e-01,3.69410783e-01]],[[-4.44886759e-02,-5.57623953e-02,4.83648717e-01],[-2.40860477e-01,-1.14619084e-01,-2.20838398e-01],[-4.76547122e-01,2.98720539e-01,-9.57804859e-01]],[[3.42991680e-01,-3.28673363e-01,-8.29893053e-01],[-3.98812771e-01,9.20739993e-02,5.46338409e-02],[-1.96803272e-01,-2.89647505e-02,3.36260915e-01]],[[-4.86764789e-01,-9.52409863e-01,7.50272721e-02],[-1.81803644e-01,6.26220942e-01,5.21990120e-01],[-2.98773319e-01,-9.72852297e-03,-8.19230378e-01]],[[1.53389394e-01,5.43628186e-02,-4.14850712e-01],[5.15222132e-01,3.67457777e-01,-4.30038184e-01],[-4.92828786e-01,9.43017960e-01,-5.57258964e-01]],[[4.20432203e-02,-3.10827158e-02,5.71007654e-02],[5.20985246e-01,-4.98226732e-01,-9.68275249e-01],[-9.96838324e-04,4.28177118e-01,1.90689355e-01]],[[-1.63239300e-01,-2.17743441e-01,-5.34476519e-01],[-1.39171137e-02,-3.26563008e-02,-2.31465593e-01],[3.22207808e-01,5.66416562e-01,-7.95154750e-01]],[[-3.51038247e-01,5.39509356e-01,-1.76641360e-01],[-7.42010236e-01,-4.44537401e-01,-7.52326429e-01],[-5.76772273e-01,-6.62016213e-01,5.86062968e-01]],[[-2.89748073e-01,2.33627893e-02,3.94061990e-02],[-3.28992188e-01,-8.75084549e-02,7.94571698e-01],[-6.41157687e-01,6.28051341e-01,-3.55962694e-01]],[[2.02276558e-01,-2.98230886e-01,4.79141057e-01],[5.95025837e-01,8.14965889e-02,1.14223082e-02],[7.39582241e-01,-7.89660215e-01,9.26187098e-01]],[[-7.90510699e-02,4.14718121e-01,-3.75038117e-01],[-7.04602972e-02,-2.85829872e-01,-2.90627182e-01],[-3.27190816e-01,2.74574280e-01,5.33556521e-01]],[[2.23598653e-03,-2.60583520e-01,2.23909155e-01],[2.33469680e-01,-8.27678107e-03,-3.23230207e-01],[9.01651561e-01,-1.53539062e+00,6.39086545e-01]],[[-1.00487661e+00,2.19009086e-01,-1.06012762e-01],[-5.17832376e-02,1.07874945e-01,-3.57779145e-01],[-1.24173403e-01,-2.27817684e-01,1.29170446e-02]],[[-1.05879903e+00,-2.52206773e-01,1.18177079e-01],[-1.08599924e-01,-4.38489802e-02,3.13910306e-01],[-2.22562388e-01,1.10315847e+00,6.83834910e-01]],[[-3.47982079e-01,-6.37270272e-01,5.06883740e-01],[-1.23484515e-01,-5.00958025e-01,-1.21820621e-01],[4.46082056e-02,4.23116207e-01,-4.04028714e-01]],[[-1.34477392e-01,1.69779137e-01,-2.68608987e-01],[4.99393731e-01,2.79880643e-01,-3.69868100e-01],[-3.00358571e-02,-1.44293174e-01,5.82819045e-01]],[[3.34454179e-01,-1.34767443e-01,-3.91482055e-01],[7.64678657e-01,3.87016118e-01,1.86955914e-01],[3.16650778e-01,-9.47879791e-01,7.02458024e-01]],[[6.10877752e-01,-8.92886817e-01,7.28584528e-02],[-2.13756531e-01,-4.16634053e-01,-3.48636031e-01],[-5.13837516e-01,5.25813460e-01,5.64485252e-01]],[[5.02431452e-01,-1.34811580e-01,5.30052334e-02],[8.75009000e-02,-1.77289605e-01,6.54181004e-01],[-2.32771054e-01,-2.75403619e-01,3.72010350e-01]],[[-6.41568303e-02,-3.08705509e-01,-6.34066939e-01],[-6.30910397e-01,-1.03614569e+00,-5.64242005e-01],[-2.18510062e-01,-4.05431002e-01,-2.76380986e-01]],[[3.97871174e-02,4.47252095e-01,8.33681881e-01],[7.21447349e-01,3.31975400e-01,-6.53058827e-01],[6.88698962e-02,2.99613595e-01,-4.07082021e-01]],[[-1.16781861e-01,8.81839871e-01,1.99601829e-01],[1.05716683e-01,2.36792684e-01,-2.97425717e-01],[3.59350681e-01,6.28765404e-01,1.04862201e+00]]],[[[-9.50068712e-01,4.88467246e-01,3.27970266e-01],[-2.51212835e-01,4.60919917e-01,1.74489498e-01],[-3.53264481e-01,3.93036813e-01,-3.36527340e-02]],[[-4.07679915e-01,1.06932007e-01,4.81121950e-02],[-2.81638950e-01,-5.66421151e-01,-1.39381230e-01],[-4.97355312e-01,7.62702465e-01,5.41748106e-01]],[[3.49194884e-01,5.54960549e-01,1.36714089e+00],[-3.15452754e-01,-7.25126803e-01,-1.66595280e-01],[2.39580482e-01,1.71456471e-01,8.50330234e-01]],[[1.38085991e-01,3.42960656e-02,-4.06490058e-01],[-5.13900936e-01,8.54122519e-01,-5.76872110e-01],[-8.40960741e-02,2.11424381e-01,-2.90061012e-02]],[[-8.39092970e-01,2.90864438e-01,-3.28353792e-01],[3.86573315e-01,8.65417570e-02,3.86934191e-01],[-6.18959606e-01,-2.68974334e-01,-5.81859648e-01]],[[1.66760996e-01,-2.42012441e-01,1.00400496e+00],[-1.67099267e-01,-2.65378296e-01,1.89011246e-01],[-8.83563995e-01,-3.15365523e-01,-2.06696749e-01]],[[-3.24580044e-01,5.41733623e-01,-4.76141721e-01],[-2.57707536e-01,-4.34762448e-01,1.17661387e-01],[-4.52161252e-01,-5.39810061e-01,-4.23827052e-01]],[[-1.53196394e-01,5.75692654e-01,-1.47301376e-01],[-2.47718871e-01,-1.06520869e-01,5.23682594e-01],[2.21761093e-01,4.36095983e-01,-7.83194378e-02]],[[-6.99749589e-01,5.20795107e-01,-4.46672179e-02],[-1.30713129e+00,-7.29303002e-01,6.45344734e-01],[-3.02284844e-02,-4.20413166e-01,-3.81321222e-01]],[[2.05932587e-01,-5.88567019e-01,-1.85515825e-02],[-1.03470182e+00,6.06164038e-01,-1.25808164e-01],[-5.41007817e-01,-9.55049992e-02,2.34109998e-01]],[[1.43729553e-01,2.39274487e-01,2.27029711e-01],[-1.83521345e-01,-2.14300901e-01,5.41786134e-01],[-8.92063007e-02,-5.53452112e-02,2.39471063e-01]],[[6.71693265e-01,-2.49264333e-02,2.78313607e-01],[-1.33962423e-01,-2.19358325e-01,3.76128823e-01],[-8.22804630e-01,-6.98356749e-03,4.86092977e-02]],[[1.38397604e-01,-5.31037748e-01,1.02814138e-02],[-1.63104281e-01,1.93430215e-01,-4.10189241e-01],[-1.04940988e-01,-7.59206176e-01,-2.04908997e-01]],[[3.69955093e-01,-5.59340894e-01,-4.23470438e-01],[-5.40793419e-01,3.82758230e-01,-5.75289369e-01],[1.22002447e+00,-2.94641286e-01,2.29783177e-01]],[[6.00507915e-01,-7.42728174e-01,-5.31651914e-01],[3.17676157e-01,-3.80033106e-02,-9.48461294e-01],[5.70146501e-01,-7.87948370e-02,6.26530647e-01]],[[-3.70797276e-01,3.31891656e-01,-8.99605826e-02],[-3.94463629e-01,4.38171655e-01,4.92569566e-01],[3.67014557e-02,-2.48269551e-02,5.81176400e-01]],[[-1.10346861e-01,-1.46843314e+00,5.47440231e-01],[2.15971425e-01,-1.87163264e-01,1.28410473e-01],[3.72972637e-01,-1.08014658e-01,1.36136070e-01]],[[-3.69715929e-01,-1.71091557e-01,3.36065531e-01],[-7.50621676e-01,6.48130000e-01,-3.36046249e-01],[1.79178417e-01,-3.14973615e-04,2.94273853e-01]],[[-1.96618140e-01,-4.92567532e-02,1.55197620e-01],[2.09889244e-02,-9.32266489e-02,4.12496984e-01],[1.09046213e-01,-4.36552078e-01,1.27941012e-01]],[[-4.03114915e-01,-8.58117193e-02,-3.94361138e-01],[-8.12982842e-02,3.62403035e-01,-1.65979359e-02],[-4.78127867e-01,-1.52521238e-01,-5.05760729e-01]],[[2.90453523e-01,4.29463565e-01,1.33330858e+00],[2.09400490e-01,5.22354484e-01,4.56366807e-01],[-3.08039431e-02,1.66253462e-01,-7.99334824e-01]],[[-8.04370284e-01,-4.32496697e-01,-4.01649117e-01],[-1.16385490e-01,6.34095430e-01,-4.92922097e-01],[2.30818525e-01,-5.73281229e-01,-1.43232550e-02]],[[-1.24520734e-01,5.40058076e-01,-8.10809970e-01],[-1.37337282e-01,1.43191889e-01,2.03156531e-01],[3.02771002e-01,-1.64142221e-01,2.90294979e-02]],[[-5.70946515e-01,-2.08444178e-01,-1.03103721e+00],[5.04952729e-01,-1.57971516e-01,2.33611718e-01],[-3.16656083e-01,1.35202378e-01,-7.28620768e-01]],[[-1.83158129e-01,-2.22876012e-01,-9.57638144e-01],[1.37097500e-02,5.55161953e-01,-8.83743405e-01],[1.76223099e-01,-3.20836604e-01,3.16412039e-02]],[[-8.29778984e-03,-2.60302961e-01,-6.84906304e-01],[4.85437125e-01,1.05196014e-01,-7.88793564e-01],[2.57227659e-01,6.47611439e-01,1.01131403e+00]],[[-1.75905496e-01,2.47606024e-01,4.41095196e-02],[6.69439077e-01,-5.67849636e-01,3.23577076e-01],[-4.24927443e-01,8.28693330e-01,-1.44717717e+00]],[[1.48736313e-02,4.38883156e-01,-3.83724838e-01],[-2.56199121e-01,4.78215694e-01,2.95973152e-01],[8.45822543e-02,7.03972816e-01,-5.85866988e-01]],[[5.46780109e-01,-1.89093620e-01,-7.75636509e-02],[6.12469725e-02,2.39332095e-01,8.34902883e-01],[-2.15269864e-01,1.57604843e-01,-5.79314053e-01]],[[4.56957281e-01,5.23042738e-01,5.00022531e-01],[3.70700121e-01,-2.97299977e-02,2.34114096e-01],[5.70734441e-01,-9.84554768e-01,5.75909078e-01]],[[5.02434075e-01,1.40501305e-01,2.38174811e-01],[-2.21516460e-01,2.30556965e-01,-3.39877188e-01],[3.40268075e-01,-2.08904311e-01,-4.69658487e-02]],[[-3.30385476e-01,-5.35974681e-01,2.05024764e-01],[-1.44863993e-01,5.20772398e-01,2.33476073e-01],[-6.52596414e-01,-3.57320517e-01,-2.92885154e-01]],[[-7.96972215e-03,1.31481439e-01,1.61768436e-01],[-3.18042934e-01,9.49697644e-02,8.39037359e-01],[-6.38346851e-01,-5.98404929e-02,4.48225945e-01]],[[-5.87174296e-01,1.13831930e-01,-2.61938423e-02],[3.35550010e-01,-3.71806979e-01,-1.24461159e-01],[-1.26527059e+00,-1.92932472e-01,-2.48054057e-01]],[[-3.38507863e-03,-2.62250334e-01,1.85737878e-01],[4.42427248e-01,7.85673320e-01,4.16559167e-02],[3.60970914e-01,-2.86260210e-02,4.72604632e-01]],[[-1.15133367e-01,-1.44385234e-01,2.78313696e-01],[6.20232075e-02,1.13796920e-01,-3.68628383e-01],[8.54522288e-02,1.90825343e-01,6.82499766e-01]],[[-2.93612957e-01,4.51243579e-01,-2.15861708e-01],[3.49584222e-01,-4.22116071e-01,-1.58185303e-01],[-4.11796480e-01,-3.83650116e-03,1.96470633e-01]],[[-7.32322216e-01,-1.34854466e-02,-7.22204268e-01],[-8.34237635e-02,-3.41142088e-01,4.54764336e-01],[1.23252399e-01,2.73861647e-01,7.96950683e-02]],[[1.02211244e-01,9.95962501e-01,-4.76906538e-01],[-5.36104202e-01,-1.14617638e-01,-1.17674291e-01],[3.62952828e-01,-8.43401775e-02,1.21190265e-01]],[[3.78169507e-01,-1.10096753e+00,5.53995311e-01],[-5.07465303e-01,-9.62799862e-02,1.74946636e-01],[-1.05829644e+00,-3.81633312e-01,-2.87211597e-01]]],[[[-8.47230732e-01,-1.01723886e+00,-3.54489952e-01],[2.45147586e-01,3.54832828e-01,-1.98826209e-01],[1.77519321e-01,-2.51212209e-01,1.20362500e-02]],[[3.57553691e-01,2.86912352e-01,-2.05774412e-01],[-1.77093014e-01,-3.47820818e-01,2.03731239e-01],[-1.11197567e+00,-3.16814706e-02,1.85302079e-01]],[[-5.20810843e-01,1.84509620e-01,3.02076072e-01],[-2.16776237e-01,8.50068778e-02,7.53915727e-01],[-3.12019497e-01,-3.15306723e-01,2.40857512e-01]],[[-4.55988228e-01,-4.60712202e-02,-1.97199499e-03],[-8.10749978e-02,8.90897572e-01,3.71557087e-01],[-3.21778432e-02,8.94728363e-01,2.95051068e-01]],[[8.55486989e-01,-3.79074961e-01,-1.30880237e+00],[-7.17107415e-01,8.78826007e-02,-7.54195392e-01],[-1.32094890e-01,-9.69008356e-02,-3.63638490e-01]],[[6.80334792e-02,1.12412803e-01,6.36259198e-01],[4.76971716e-01,-4.15978469e-02,5.40826976e-01],[6.44647107e-02,2.64791638e-01,5.63235760e-01]],[[-1.76918849e-01,-5.15065014e-01,-3.13967168e-01],[-5.13157725e-01,-1.65948957e-01,-3.36954832e-01],[-2.11086512e-01,7.24879131e-02,-1.42988518e-01]],[[-4.94846135e-01,1.52909413e-01,-1.09280072e-01],[3.22271913e-01,-7.17983007e-01,-3.31456840e-01],[1.48063028e+00,-8.59351158e-02,2.25426093e-01]],[[-3.18781376e-01,-4.37015980e-01,2.05729976e-01],[1.06117272e+00,7.64179789e-03,2.48525977e-01],[-8.82058918e-01,-2.48172492e-01,-3.24980021e-01]],[[1.42236769e-01,1.77895218e-01,2.18827173e-01],[-1.57349244e-01,3.78085554e-01,-5.46283722e-02],[2.00353861e-02,-3.44831884e-01,-3.11425090e-01]],[[-4.13865805e-01,-2.39118151e-02,-6.51245117e-02],[-5.28935373e-01,2.70750940e-01,-8.55327070e-01],[4.13141102e-02,-3.58293265e-01,-8.26109529e-01]],[[5.88437140e-01,-6.77747548e-01,-9.61143970e-02],[1.37304170e-02,-2.38145009e-01,-2.66459614e-01],[-2.75426686e-01,-2.79046476e-01,3.75881702e-01]],[[-6.00720383e-02,-6.07263520e-02,-2.32989974e-02],[-5.12461722e-01,5.49697399e-01,7.57234037e-01],[-8.64066064e-01,-2.01961011e-01,-6.70131624e-01]],[[2.08522454e-01,1.05287707e+00,1.53465211e-01],[3.34529966e-01,4.65508789e-01,1.06087632e-01],[2.30030436e-02,-1.71583384e-01,6.37666762e-01]],[[1.99584916e-01,-3.78613085e-01,-9.89213958e-03],[-4.53405946e-01,-1.98701262e-01,-2.49231532e-01],[-2.44828567e-01,2.91362584e-01,-5.45278609e-01]],[[-4.42714505e-02,-6.38401031e-01,4.44400042e-01],[4.59092975e-01,-7.90463667e-03,-2.29526758e-01],[-4.45212424e-01,2.74876118e-01,-1.02190292e+00]],[[-4.35176879e-01,-2.77119428e-01,2.64942288e-01],[-1.15991557e+00,2.49229744e-01,-3.08749318e-01],[-8.91880333e-01,5.69198251e-01,-5.87301433e-01]],[[-1.28517136e-01,-2.96461284e-01,1.44974262e-01],[3.25320423e-01,2.11247563e-01,-6.14524186e-01],[-4.88692448e-02,-1.28050610e-01,-4.32670534e-01]],[[8.89901668e-02,7.94583559e-02,2.70417064e-01],[-2.12563097e-01,1.94236293e-01,6.36108696e-01],[-1.06129086e+00,2.06721067e-01,-6.91961110e-01]],[[-6.10217392e-01,5.33724964e-01,-4.85313743e-01],[6.67077899e-01,2.23244846e-01,-4.97670799e-01],[-1.80135310e-01,3.32146347e-01,5.28702676e-01]],[[4.13666904e-01,-1.78386882e-01,5.99256456e-02],[-4.22120869e-01,1.03582978e+00,-1.85949504e-01],[2.96153761e-02,-1.30522296e-01,-7.44269788e-01]],[[-6.89917266e-01,3.19519639e-03,1.37600765e-01],[1.89815879e-01,-8.03725719e-01,-2.86777336e-02],[-4.21142071e-01,-4.67236608e-01,1.10675797e-01]],[[3.39521945e-01,7.52274215e-01,1.18856180e+00],[6.05956674e-01,-8.97327065e-01,4.57092643e-01],[1.00187324e-02,-7.96589077e-01,6.77906990e-01]],[[2.88490236e-01,1.14292055e-01,-3.45802516e-01],[-1.70382559e-01,-4.70127881e-01,6.36993051e-01],[-2.27880090e-01,3.03567380e-01,3.98817696e-02]],[[-5.45525014e-01,2.10331351e-01,2.67159790e-01],[5.35812750e-02,-3.25953573e-01,-9.93096083e-02],[-3.19852591e-01,4.81467843e-02,9.60879266e-01]],[[-9.67736766e-02,-7.05081746e-02,-2.18168825e-01],[3.88416439e-01,-4.57761765e-01,2.27460619e-02],[3.38760197e-01,-3.57333302e-01,2.06719950e-01]],[[-2.55953848e-01,1.92643613e-01,6.00316823e-01],[-1.94447607e-01,4.31161612e-01,2.37759605e-01],[3.09885710e-01,2.35151842e-01,3.09073776e-01]],[[1.89117804e-01,-4.54552621e-01,-1.63998127e-01],[-2.61950552e-01,-2.67797738e-01,-1.52936861e-01],[-1.60805598e-01,-7.74708748e-01,1.39496148e-01]],[[7.72529235e-03,1.54157251e-01,-6.02488697e-01],[-8.56666207e-01,6.14299953e-01,-1.37503698e-01],[-9.56643343e-01,-3.94629419e-01,-2.21419573e-01]],[[6.09362066e-01,5.74501455e-02,9.47899163e-01],[6.16149902e-01,-1.05132563e-02,3.49069148e-01],[-8.61013159e-02,-3.07005197e-01,2.19156612e-02]],[[-5.90300798e-01,3.91987532e-01,6.35654390e-01],[-2.01767117e-01,7.08642695e-03,1.81668252e-01],[-1.06510210e+00,-4.81829524e-01,-8.39962345e-03]],[[-4.68287170e-02,3.13747555e-01,5.34726121e-02],[-9.27023351e-01,-3.89968276e-01,-2.61482894e-01],[1.18522853e-01,5.79221368e-01,1.03307933e-01]],[[-1.44511864e-01,7.82786161e-02,-2.10297182e-01],[3.88764828e-01,-3.35831344e-01,1.69085339e-01],[6.71975434e-01,-4.16930825e-01,6.31857365e-02]],[[-1.85076058e-01,3.95485193e-01,-3.84615839e-01],[-4.21576574e-02,-5.36013320e-02,-1.10936975e+00],[-7.97847867e-01,4.21005577e-01,4.57089134e-02]],[[3.19678992e-01,-5.58811247e-01,5.27292013e-01],[6.91410825e-02,3.83197576e-01,-3.80580723e-01],[2.21058398e-01,-2.37051859e-01,-8.97731602e-01]],[[-3.44773172e-04,4.07549381e-01,-3.71284127e-01],[2.00518280e-01,6.51782528e-02,2.07703695e-01],[-2.19787046e-01,-1.79382265e-01,-2.88374186e-01]],[[-3.54318708e-01,9.79289889e-01,-4.48920935e-01],[3.28511953e-01,1.77312151e-01,3.72311234e-01],[5.15606463e-01,7.42355049e-01,-1.53211147e-01]],[[-1.28069475e-01,4.46927965e-01,1.99131891e-01],[2.64982015e-01,-2.94974744e-01,3.26130122e-01],[3.21814716e-01,-2.61363655e-01,3.43823344e-01]],[[6.75054729e-01,-3.71998921e-02,-6.61446825e-02],[4.76711750e-01,1.03000738e-01,6.10599816e-01],[3.79409157e-02,-2.65376419e-01,5.09419382e-01]],[[4.51036602e-01,7.54116178e-02,3.29704732e-01],[-3.25022101e-01,-4.01355535e-01,6.57091737e-01],[-8.73150676e-03,8.61863673e-01,-5.85167944e-01]]],[[[-9.01993692e-01,3.92618449e-03,2.15406165e-01],[-5.74009478e-01,1.68984190e-01,1.25817657e-01],[3.69224221e-01,7.32136071e-02,3.18666965e-01]],[[6.07639551e-02,2.60314137e-01,-2.53880203e-01],[-2.99445182e-01,2.76816189e-01,-1.05182223e-01],[-7.40775347e-01,-4.02824461e-01,-1.30081102e-01]],[[2.95689367e-02,-8.14554930e-01,8.86560440e-01],[-4.89215478e-02,-4.63424891e-01,4.03944105e-01],[-1.54353753e-01,2.60498405e-01,8.31421375e-01]],[[1.15603864e-01,3.95553380e-01,4.59551632e-01],[-4.60344464e-01,-2.87087858e-01,-7.78143108e-01],[-9.15201455e-02,-8.03937018e-01,2.84779638e-01]],[[-2.89190203e-01,-1.37402847e-01,-6.27568483e-01],[2.76192188e-01,2.55887330e-01,6.72949702e-02],[2.50743598e-01,2.10284486e-01,-1.66594177e-01]],[[2.99248904e-01,4.40989494e-01,-7.51638591e-01],[8.23229551e-02,-5.08114457e-01,6.63847506e-01],[-4.70787197e-01,-5.13480484e-01,-3.97831686e-02]],[[4.56370980e-01,-8.96276474e-01,4.19539452e-01],[-9.44502950e-02,1.27388909e-01,1.09620437e-01],[-2.57289391e-02,-2.06367537e-01,-4.77402717e-01]],[[-1.35701552e-01,-7.34455466e-01,3.35186183e-01],[-2.27398843e-01,6.84773445e-01,4.66727793e-01],[-7.41136372e-01,1.84552699e-01,-1.06642938e+00]],[[8.45811844e-01,3.71542543e-01,5.32291651e-01],[4.22616780e-01,3.42418343e-01,8.52116570e-02],[4.10093397e-01,5.44648707e-01,-1.32667562e-02]],[[4.07763511e-01,-9.54209208e-01,-7.50517175e-02],[7.35393763e-01,4.69555467e-01,-1.02503307e-01],[8.71226788e-01,-3.61255139e-01,-3.02441210e-01]],[[-3.93038355e-02,1.57568008e-01,2.39277840e-01],[-2.12335542e-01,-5.93058705e-01,-1.70250759e-02],[-4.73653153e-02,-5.93130708e-01,-1.15010762e+00]],[[-7.02222943e-01,-1.50220143e-02,6.93294823e-01],[3.50857496e-01,7.10988790e-02,1.38555437e-01],[3.09829921e-01,1.35203905e-03,-5.79953101e-03]],[[7.71715492e-02,-2.42774785e-01,-5.19199312e-01],[-9.50990692e-02,3.85783464e-01,-1.44838974e-01],[-1.18901646e+00,3.85400325e-01,-9.65377316e-02]],[[4.92977798e-01,-3.15774488e-03,-1.14434734e-01],[7.30446950e-02,9.27005291e-01,-3.35742205e-01],[-6.09013736e-01,9.02335718e-02,-4.75068688e-01]],[[-3.42333376e-01,-1.08633474e-01,-1.93987146e-01],[-4.45990544e-03,-3.69362980e-01,-7.28784144e-01],[-6.63275793e-02,6.76542759e-01,6.27230406e-01]],[[1.18569985e-01,-4.27878469e-01,-4.43509996e-01],[-4.85230833e-01,-3.39007378e-02,-4.81873631e-01],[-1.32588908e-01,-6.08005762e-01,-6.65738225e-01]],[[3.02513927e-01,-2.10433573e-01,1.28553472e-02],[3.87049288e-01,-1.62148297e-01,-1.97557241e-01],[6.83816135e-01,2.17644066e-01,1.62589371e-01]],[[5.79520106e-01,6.27486765e-01,5.37086427e-02],[4.56047267e-01,-1.06790280e+00,-5.08133829e-01],[-5.13220549e-01,5.90227008e-01,2.84657806e-01]],[[1.79990813e-01,1.43373728e-01,1.92939602e-02],[3.43622863e-01,1.10464834e-01,7.07392454e-01],[-1.49199820e+00,3.10184658e-01,2.84182131e-01]],[[-6.88317344e-02,2.72792816e-01,-6.22529805e-01],[-2.36579441e-02,-2.02902123e-01,-5.87042928e-01],[9.13432762e-02,7.38054156e-01,7.76568413e-01]],[[1.63428128e-01,-5.66248834e-01,-3.34842592e-01],[6.30494714e-01,-3.24128091e-01,2.54796088e-01],[-4.04584408e-01,-3.72094624e-02,2.32939199e-01]],[[-5.93314588e-01,8.16221014e-02,-2.80175716e-01],[4.20371264e-01,2.14438051e-01,1.35740191e-01],[-2.29908571e-01,-5.55614114e-01,5.69616318e-01]],[[2.49343961e-01,-1.43479019e-01,1.35871994e+00],[-2.98834413e-01,-1.58570901e-01,2.31467783e-01],[-2.59733051e-01,5.48754856e-02,-6.60065711e-01]],[[3.09101760e-01,-6.56684577e-01,3.95849980e-02],[2.79958814e-01,-1.17549114e-01,1.48881328e+00],[-9.57182273e-02,2.74082661e-01,6.05291985e-02]],[[-3.74755077e-02,4.98980939e-01,-9.16170776e-02],[3.02774370e-01,-2.50677824e-01,-2.60441452e-01],[3.35825473e-01,-1.35506794e-01,3.94772708e-01]],[[-2.23514929e-01,-2.40553632e-01,7.60115027e-01],[7.48074710e-01,-4.49363053e-01,1.45264268e-01],[-2.24641472e-01,-3.16492736e-01,-2.08764225e-01]],[[2.77931560e-02,-3.19387734e-01,-9.96538401e-01],[2.48554666e-02,-4.55312967e-01,3.18088681e-01],[-3.76886204e-02,3.81993830e-01,-4.46280763e-02]],[[2.33383998e-01,-4.45996940e-01,-1.87779665e-01],[-2.00485051e-01,-3.31232756e-01,5.79623222e-01],[3.58572006e-01,-8.96073654e-02,1.84777215e-01]],[[-5.31509817e-01,-2.06025243e-01,4.34894674e-02],[2.00559780e-01,-4.06814925e-02,1.92404807e-01],[3.50840926e-01,-6.70275018e-02,-6.26103878e-01]],[[-4.74980712e-01,2.01280966e-01,-8.99924245e-03],[-6.25561357e-01,-3.65642369e-01,-5.87802986e-03],[-4.90199149e-01,-8.54229212e-01,-7.58990884e-01]],[[-1.12589404e-01,-2.01755445e-02,-4.59551156e-01],[-6.83355033e-01,-6.06160283e-01,2.52878994e-01],[-1.87888816e-01,1.92765400e-01,5.55952452e-02]],[[1.52462125e-01,6.98729008e-02,1.64041817e-01],[-3.14559877e-01,-2.02480748e-01,4.64261353e-01],[-4.42295611e-01,-7.20850885e-01,2.98228174e-01]],[[4.81482506e-01,1.69122770e-01,-2.56929606e-01],[5.00056565e-01,2.15764761e-01,1.49073556e-01],[1.37435243e-01,-3.32885176e-01,-8.23711976e-02]],[[-3.61128412e-02,-7.55313516e-01,4.63655323e-01],[2.50042021e-01,2.48843744e-01,-5.51207900e-01],[-2.36067533e-01,3.03927720e-01,-7.70092336e-03]],[[1.42066047e-01,-7.62566984e-01,4.80981320e-01],[-3.66966516e-01,-6.97569251e-01,-1.32049397e-01],[-6.17707253e-01,-1.40052959e-01,-1.52098703e+00]],[[-1.00528605e-01,-1.29025122e-02,-4.75375447e-03],[-5.58990717e-01,-8.62176046e-02,1.36300996e-01],[4.77364995e-02,4.12904143e-01,-1.38589132e+00]],[[-4.34868127e-01,-3.51710677e-01,5.60752332e-01],[-1.45860970e-01,-6.46583796e-01,5.10609686e-01],[-2.25540429e-01,-9.90937278e-02,-5.37023604e-01]],[[-6.77433372e-01,-9.89176095e-01,-6.97963357e-01],[2.59836406e-01,-6.79537117e-01,5.60823321e-01],[5.43126881e-01,-1.99754059e-01,-1.97219357e-01]],[[-4.98300016e-01,3.26658875e-01,-2.91719228e-01],[-1.92142010e-01,-6.00552797e-01,-1.64993867e-01],[1.96240544e-01,-3.28925967e-01,8.77792854e-03]],[[4.05870140e-01,-4.91611838e-01,-1.22015834e-01],[3.24618518e-01,1.77213505e-01,2.42076084e-01],[-4.32685800e-02,-1.80382747e-02,-4.94644105e-01]]],[[[-1.67133495e-01,-3.30772012e-01,6.47913635e-01],[-2.22041249e-01,2.95592900e-02,-3.68949100e-02],[-7.03483298e-02,-2.05200195e-01,7.16525197e-01]],[[2.59202063e-01,4.98606324e-01,5.33474386e-01],[-3.29157531e-01,-1.28552571e-01,1.31460220e-01],[1.54035538e-01,4.65336889e-01,-1.32684588e-01]],[[-1.23637772e+00,6.15699768e-01,1.48070067e-01],[1.27481103e-01,2.79043555e-01,2.60172784e-01],[-2.80642677e-02,-1.42421909e-02,-6.74603805e-02]],[[-3.01149786e-01,-7.26925954e-02,9.29084122e-02],[-4.02097683e-03,2.47382507e-01,-5.32888532e-01],[-1.07553765e-01,7.23451138e-01,-4.25021142e-01]],[[-3.77969109e-02,3.05115163e-01,1.99863791e-01],[-2.20779076e-01,-2.99716979e-01,2.13869050e-01],[1.32035941e-01,-2.07820550e-01,2.79814266e-02]],[[-2.30147898e-01,5.24865448e-01,2.17569202e-01],[-9.41571236e-01,1.16113417e-01,6.64906859e-01],[2.25705683e-01,6.15780689e-02,-1.01557803e+00]],[[-1.59262881e-01,2.03978613e-01,-3.69271100e-01],[3.10814500e-01,-4.27558571e-01,4.78588343e-01],[2.95872539e-01,3.25401098e-01,-7.63844728e-01]],[[-3.67935538e-01,3.37240659e-02,6.61318541e-01],[-4.69983310e-01,-2.18242630e-01,5.77200532e-01],[5.23099422e-01,-2.81016499e-01,7.12809106e-03]],[[-1.49117902e-01,4.59882766e-01,8.13309848e-01],[3.60599607e-01,4.39558238e-01,8.32459688e-01],[3.63882571e-01,4.01409537e-01,4.27125618e-02]],[[-5.18120706e-01,-7.28841126e-01,-2.29492798e-01],[-2.25269526e-01,5.44152856e-01,-4.50497299e-01],[-8.70446980e-01,-3.94138545e-01,6.66366160e-01]],[[-8.03710967e-02,-6.82495236e-01,-9.41412926e-01],[-1.02043092e+00,4.74171154e-03,-2.83253759e-01],[-4.97797579e-01,-8.32692564e-01,-2.52617747e-02]],[[1.65242597e-01,7.14718640e-01,2.74101138e-01],[1.59142911e-01,-2.90299088e-01,-7.48738408e-01],[3.60196233e-02,2.99109906e-01,8.06250393e-01]],[[3.65998060e-01,-3.52954954e-01,1.77326903e-01],[2.07482919e-01,-4.83239889e-01,2.25270446e-02],[-1.39412358e-01,5.77565014e-01,1.47248030e-01]],[[3.89295042e-01,1.06320910e-01,-5.28422184e-02],[3.43754768e-01,-5.10161161e-01,2.06946120e-01],[1.27605057e+00,2.68885583e-01,2.42835954e-01]],[[1.93841726e-01,1.45017788e-01,-1.68723091e-02],[-3.98805022e-01,2.06887349e-01,1.04964606e-01],[1.56002477e-01,-4.16393459e-01,3.56043488e-01]],[[-4.47250307e-01,-1.79179281e-01,-1.09528804e+00],[-7.30154514e-01,3.87076348e-01,-2.19954792e-02],[2.23243773e-01,-8.33602965e-01,1.23770244e-01]],[[3.05182636e-01,2.54619986e-01,-6.54465035e-02],[-1.21375851e-01,2.52579689e-01,7.96868876e-02],[-3.75745714e-01,-5.69909632e-01,-3.38387668e-01]],[[5.43578565e-01,1.14749886e-01,3.23186785e-01],[-1.71829119e-01,4.51113015e-01,4.05473053e-01],[7.78287292e-01,8.46077874e-02,-8.87841761e-01]],[[8.56790245e-02,2.38573298e-01,-7.30362177e-01],[-5.51832914e-01,-1.93769082e-01,-1.39895109e-02],[1.34137049e-01,-3.93031478e-01,6.21894658e-01]],[[-6.53841719e-02,-1.32027671e-01,5.56056276e-02],[-1.14490464e-01,2.94734061e-01,-1.41049609e-01],[2.47495770e-01,1.12352215e-01,-2.24381462e-01]],[[-4.77867499e-02,2.24040523e-01,-1.34616405e-01],[7.22712338e-01,8.50123987e-02,-5.48112035e-01],[4.72353339e-01,2.35288262e-01,-5.32353342e-01]],[[-1.23714244e+00,-1.31154031e-01,-5.67226768e-01],[-1.55252695e-01,-4.74243224e-01,3.31101477e-01],[8.64544749e-01,2.56604165e-01,4.88358021e-01]],[[-1.03309608e+00,-9.14009273e-01,-2.38113806e-01],[4.59114611e-02,-3.64592165e-01,2.67673373e-01],[9.18804705e-01,-2.83601992e-02,8.06650743e-02]],[[5.53896129e-01,1.47981271e-01,-5.34587502e-01],[2.58893460e-01,-5.64620733e-01,3.02017212e-01],[2.00683206e-01,1.44192785e-01,2.01189250e-01]],[[-7.83179760e-01,3.54256362e-01,-3.64340544e-01],[4.63531047e-01,-6.02571845e-01,1.40166178e-01],[-8.37815583e-01,-1.25251830e-01,-2.25781798e-01]],[[2.57299811e-01,-2.26575345e-01,1.45418225e-02],[-2.53183424e-01,5.16314387e-01,-1.23750515e-01],[4.16239947e-01,-4.73662078e-01,1.02808453e-01]],[[8.13502669e-01,-8.59057680e-02,-1.87617138e-01],[7.11929798e-01,7.24066734e-01,-2.30270639e-01],[-4.93142843e-01,-3.28278452e-01,-1.46317899e+00]],[[8.62226665e-01,-3.37425143e-01,-1.21867275e+00],[3.57747942e-01,4.14878845e-01,2.54313145e-02],[3.99312466e-01,-2.76522756e-01,2.07061455e-01]],[[2.87694752e-01,2.81042792e-03,3.86480361e-01],[4.43933994e-01,5.09185016e-01,2.06284210e-01],[-4.47695285e-01,-9.14396122e-02,5.76244891e-01]],[[2.09047496e-01,-1.50293767e-01,-5.55838421e-02],[8.26832354e-02,3.88463214e-02,-1.16957754e-01],[3.70069385e-01,4.15821001e-02,-7.12925121e-02]],[[-8.52298066e-02,-1.01536915e-01,-5.39855599e-01],[7.82364309e-01,7.58087710e-02,-3.54789257e-01],[5.34275651e-01,-1.25563717e+00,3.86161841e-02]],[[-4.95606601e-01,-3.17480266e-02,-2.91399080e-02],[9.68826234e-01,-1.97497129e-01,-4.02538091e-01],[-2.45582670e-01,-2.32462972e-01,3.95557553e-01]],[[-1.85149208e-01,6.92382216e-01,-9.62285340e-01],[-3.82041968e-02,-5.92403896e-02,-8.22499692e-01],[3.58442605e-01,-6.15163267e-01,5.90221643e-01]],[[2.15056837e-01,-3.80769044e-01,-8.92382935e-02],[-1.72768965e-01,2.72200465e-01,-6.88913986e-02],[1.44338712e-01,1.44738123e-01,-7.85206318e-01]],[[3.05316836e-01,2.84980536e-01,4.86096293e-02],[-8.65714103e-02,-5.92229426e-01,3.56993586e-01],[-1.31559506e-01,-1.24516201e+00,-7.99774826e-02]],[[3.57541114e-01,-1.83297962e-01,-5.68522036e-01],[-6.41267002e-01,5.56352735e-01,2.29059160e-02],[3.09275299e-01,-4.34592575e-01,-9.44141299e-04]],[[6.23955667e-01,9.39013734e-02,3.47585320e-01],[3.13369125e-01,-3.15699816e-01,-2.21604660e-01],[-4.38328907e-02,-3.94423515e-01,-2.48813331e-01]],[[-1.03315461e+00,1.20653860e-01,7.60040581e-01],[3.96806777e-01,4.72901165e-02,6.96375489e-01],[-1.37840688e-01,-2.33162105e-01,1.34461448e-02]],[[9.39337194e-01,-1.55519217e-01,-2.90447116e-01],[4.98374164e-01,-1.07750607e+00,5.28729670e-02],[2.63059646e-01,-1.78930350e-02,3.07124078e-01]],[[-3.02509815e-01,-2.96109021e-01,-1.50684237e-01],[-3.73638123e-01,1.17824249e-01,-1.43484138e-02],[-1.21444499e+00,2.39986349e-02,4.50862676e-01]]],[[[-8.27465057e-01,-7.17259586e-01,-2.20940099e-04],[6.14120364e-02,-1.73378885e-01,6.58825159e-01],[4.62623209e-01,1.06012449e-01,-6.97791874e-01]],[[-8.61631870e-01,-2.25542128e-01,6.87985361e-01],[-2.33146146e-01,-1.50687873e-01,7.76915997e-02],[1.10270113e-01,3.90493125e-02,-1.46420933e-02]],[[6.02815807e-01,-9.73227918e-01,-2.93702096e-01],[7.97524769e-03,-4.65840995e-01,5.35615861e-01],[-3.80868971e-01,-4.57840115e-01,1.69471785e-01]],[[-8.92236680e-02,-1.76341116e-01,5.61821401e-01],[3.52852255e-01,-6.35538995e-01,1.37765124e-01],[-6.59691870e-01,2.68684357e-01,6.95184320e-02]],[[-8.96947324e-01,-1.01680025e-01,1.75274238e-01],[1.98211551e-01,-4.34238404e-01,-3.18568438e-01],[-1.07676971e+00,-5.13443112e-01,-4.51234341e-01]],[[-4.50708777e-01,-8.57450068e-01,-3.77444953e-01],[-1.10666573e+00,1.21117942e-02,1.53557494e-01],[-4.23195846e-02,6.13459289e-01,-4.49694067e-01]],[[3.66814077e-01,1.32116050e-01,2.30082676e-01],[4.30356860e-02,4.50288683e-01,-4.01955873e-01],[7.19280720e-01,-7.92562366e-01,-8.27384889e-01]],[[7.18549490e-01,5.07858515e-01,-3.40839207e-01],[-3.95175010e-01,3.28954697e-01,-2.83224225e-01],[9.46664393e-01,2.41949633e-01,2.07975343e-01]],[[-3.27004224e-01,-1.61074370e-01,6.49677217e-02],[-3.63654286e-01,-5.22111714e-01,5.06012499e-01],[3.45273167e-01,1.58245355e-01,1.56042114e-01]],[[3.79254431e-01,-5.36123157e-01,-5.28567374e-01],[7.23353103e-02,1.04622453e-01,-2.09287271e-01],[1.28322259e-01,4.32947993e-01,-6.34248018e-01]],[[3.26101556e-02,4.99844700e-01,7.22561777e-02],[2.46764973e-01,1.20654322e-01,-4.36801851e-01],[-2.61530519e-01,1.96869269e-01,5.06257892e-01]],[[5.05758822e-01,3.70639831e-01,-3.54213089e-01],[1.57057360e-01,-1.45610273e-01,-3.11476469e-01],[-4.42419685e-02,1.52437612e-01,3.97113012e-03]],[[-1.12030494e+00,-3.48078579e-01,4.84521031e-01],[-2.23556966e-01,-4.15072531e-01,7.25930810e-01],[-6.09629512e-01,-2.26239607e-01,1.90677390e-01]],[[-3.49803954e-01,9.38387662e-02,-2.79064208e-01],[-4.47779447e-01,2.28117108e-01,3.80922966e-02],[-1.16354489e+00,-5.38605154e-01,-2.77942866e-01]],[[1.73547238e-01,2.15909973e-01,-2.82512128e-01],[-5.39836347e-01,-4.80471611e-01,-3.77746001e-02],[-8.43013108e-01,-2.21293226e-01,7.73528457e-01]],[[1.80187002e-01,-2.02479064e-02,-5.03744602e-01],[3.85736346e-01,7.93772116e-02,-4.78027284e-01],[-3.93346995e-01,-1.50676906e-01,-3.33340973e-01]],[[-1.43515915e-01,1.23527095e-01,-3.08083802e-01],[-1.20401837e-01,3.70935172e-01,3.41250420e-01],[6.73287153e-01,-1.76758900e-01,1.00561786e+00]],[[-4.33704406e-01,7.56470025e-01,-7.04177320e-01],[2.12975770e-01,-5.33628948e-02,-6.01802051e-01],[-7.62973130e-02,4.28821564e-01,-4.16061670e-01]],[[5.43928266e-01,1.70364901e-02,4.30579782e-01],[-4.93680179e-01,-2.42112190e-01,-2.12508645e-02],[-5.24825513e-01,6.47657692e-01,5.42308331e-01]],[[-2.04721197e-01,-4.52585332e-02,-3.37577403e-01],[5.56695834e-02,-8.95686597e-02,2.40966767e-01],[1.69357583e-01,-3.45321268e-01,1.18566811e+00]],[[-1.92512751e-01,-7.74708748e-01,-1.21481454e+00],[-9.67168063e-02,-3.71204555e-01,-5.49199879e-01],[3.03793341e-01,5.04778266e-01,1.34455383e-01]],[[5.39874613e-01,1.18680932e-01,1.09001413e-01],[6.36492312e-01,2.03489661e-02,-2.59556413e-01],[-2.14985162e-01,7.06220418e-03,1.24453694e-01]],[[5.84841192e-01,2.00142458e-01,6.60105348e-01],[1.43429756e-01,-9.49721858e-02,-3.15508306e-01],[-1.16481204e-02,-7.80101791e-02,5.25646210e-01]],[[-5.84637225e-01,3.92686397e-01,-3.31170142e-01],[-5.03716767e-01,-5.31654656e-02,-7.03070104e-01],[-8.59797001e-01,4.87307250e-01,-1.13348670e-01]],[[9.09282148e-01,2.36998305e-01,1.14112966e-01],[2.19710201e-01,-3.66367280e-01,1.95741847e-01],[4.36314076e-01,-6.21461391e-01,6.06021404e-01]],[[3.65874201e-01,-1.05651833e-01,-1.30043760e-01],[2.10322455e-01,6.66943252e-01,2.27967918e-01],[-1.22841454e+00,1.10222924e+00,2.32282668e-01]],[[4.91002798e-01,3.78161341e-01,-3.90053261e-04],[-5.29139340e-01,4.31472391e-01,1.58812866e-01],[-5.06063141e-02,-6.52529955e-01,4.48539823e-01]],[[6.73321128e-01,-1.89017192e-01,4.09978539e-01],[6.40382469e-01,1.48706045e-02,3.76423031e-01],[-2.41446182e-01,7.86688924e-01,7.05387056e-01]],[[-7.16166377e-01,-3.68212610e-01,-9.66957882e-02],[-2.68345233e-02,5.82270930e-03,-3.55125129e-01],[4.23554152e-01,6.74037561e-02,6.14505075e-02]],[[-7.15800822e-01,7.32162893e-02,-8.17848966e-02],[-4.93624598e-01,-1.78642318e-01,-1.05849348e-01],[-5.52782416e-01,-1.03847158e+00,4.82475370e-01]],[[-5.80784380e-01,1.14142820e-01,2.37260967e-01],[-6.21968269e-01,1.98631480e-01,1.86445937e-02],[-2.48072427e-02,2.55816579e-01,-1.17266446e-01]],[[-4.02292818e-01,-7.35100731e-02,-1.15371950e-01],[-1.81974456e-01,5.10061264e-01,-1.21465705e-01],[-1.69707194e-01,-4.15788144e-01,1.06110346e+00]],[[-1.97336391e-01,-2.37601131e-01,7.05963792e-03],[8.61771479e-02,-3.82135101e-02,-2.33294636e-01],[3.58501375e-02,2.13787466e-01,-3.00305426e-01]],[[6.40439868e-01,-9.29997414e-02,-4.41910446e-01],[3.11632315e-03,5.28126121e-01,2.22068563e-01],[3.80956411e-01,4.14842397e-01,-5.82270980e-01]],[[-3.14599752e-01,-8.47514093e-01,1.47518396e-01],[-4.31538314e-01,-3.73850614e-01,-4.75910455e-01],[-4.78476346e-01,3.62794429e-01,-6.58391416e-01]],[[5.02540529e-01,9.92662907e-02,-5.74161530e-01],[-2.05492467e-01,8.18988308e-02,-2.40147233e-01],[-3.54160845e-01,4.77410704e-01,-8.39999020e-01]],[[-1.61682256e-02,-2.77851313e-01,-3.53983790e-01],[-5.97752213e-01,1.29449725e-01,-4.06614840e-01],[-1.35359392e-01,-6.04233682e-01,5.57507217e-01]],[[-2.89872233e-02,1.11449927e-01,-1.41288817e-01],[-7.34589398e-01,-4.50369060e-01,5.69462419e-01],[1.50673106e-01,1.85202003e-01,5.29042304e-01]],[[-6.45368278e-01,-3.94030958e-01,8.45314935e-02],[2.72446964e-03,9.26444829e-02,1.20778635e-01],[1.97782338e-01,6.65516138e-01,-4.72793609e-01]],[[2.71723270e-01,-8.89647782e-01,-4.74453807e-01],[4.00176719e-02,-9.83829021e-01,-4.17150199e-01],[6.03918731e-02,-5.58085693e-03,-3.98463845e-01]]],[[[0.41545656,0.617038,0.15831064],[0.88573533,-0.02736915,-0.2650234],[0.20377368,-0.24214004,0.1727872]],[[0.00545878,-0.60180956,-0.03669535],[0.33301824,0.32388255,-0.605322],[0.5349109,0.0247299,-0.33429202]],[[0.2375111,0.44469962,-0.76511925],[-0.11660903,-0.4897652,-0.19044109],[-0.20478377,-0.34419307,-0.15894498]],[[0.14587407,-0.06549368,-0.46020794],[0.11705478,-0.6009143,0.31786254],[-0.5345504,-0.49101135,0.5312702]],[[0.09222982,-0.09299924,0.15574828],[0.01455036,-0.48874584,0.29809624],[-0.28465357,0.07207535,0.39011845]],[[0.24775241,-0.24016164,1.2424357],[0.13217121,-0.14002848,0.290943],[-0.5520463,0.22337994,-1.1835871]],[[-0.00482408,-0.18141618,0.10243689],[-0.52673763,-0.0707391,-1.0815357],[-0.86385334,0.8716485,-0.35204]],[[-0.749599,0.19661672,-0.38850573],[-0.41658056,-0.5444302,0.01360277],[-0.69436073,-0.48783413,-0.15833831]],[[-0.23300302,-0.05340158,-0.57262653],[0.00314937,0.202026,0.26064318],[-0.31926912,0.49507904,0.6935479]],[[0.1392123,0.5484116,-0.2594785],[-0.12583406,-0.3291763,0.4239928],[-0.97076386,-0.30376565,-0.01077255]],[[0.2621412,-1.090082,0.39099142],[-0.9692697,-1.0768393,0.7737355],[0.16665564,-0.06027629,0.42193624]],[[0.29217112,0.00394101,-0.29520655],[0.6315167,0.37791216,0.658486],[-0.5254448,-1.0466214,0.15160884]],[[-0.23070672,0.23779276,0.17481382],[-0.03473747,-0.62825096,0.2928345],[-0.02829662,-0.26349846,0.46613535]],[[-0.29785123,-0.536783,0.03971196],[0.02634358,-0.34057915,-0.051777],[-0.2794102,0.24196222,0.01108607]],[[0.2576692,-0.29584154,0.02183018],[-0.08719543,1.046462,-0.45311454],[0.6656613,-0.39776194,0.12533778]],[[-0.2586238,-0.00188826,0.29552302],[-0.39742452,0.00178637,0.18561555],[0.65796125,-0.70672226,0.12555818]],[[0.36999485,0.40517473,0.47963986],[0.03333116,-0.41876224,-0.36222407],[-0.13238959,0.93762416,-0.19632304]],[[0.22044304,0.11940923,-0.70969516],[-0.49135217,0.1568655,0.5703295],[-0.26245713,0.18453345,0.09611599]],[[-0.2083638,0.21988656,0.0959346],[0.31937036,0.6273219,-0.4548423],[0.14711824,0.49865043,-0.81004524]],[[0.514828,0.52396375,-0.19259469],[-0.5251972,0.18101154,-0.35632342],[-0.0035078,0.05509973,-0.3149472]],[[0.10862049,0.12680344,0.05480757],[0.8782967,-0.4237128,0.0718706],[0.7332762,-0.4035617,-0.37401757]],[[-0.4031756,0.14788033,0.40907463],[0.25734445,0.0548175,-0.41231593],[-0.03875841,0.71909934,-0.39567646]],[[-0.9346835,-0.9120037,-1.3398824],[-0.667307,0.77074987,-0.18587217],[0.1490518,-0.4472395,-0.771928]],[[-0.6692717,0.33572093,0.6653328],[0.43660453,0.40955707,-0.28761792],[0.20583397,-0.15493402,-0.5210766]],[[-0.13019873,0.11060318,-0.44008029],[0.00177135,0.6328045,-0.00445722],[-0.7109322,-0.2449948,-0.41874292]],[[0.20767705,0.4786119,-0.6987719],[0.0655421,-0.3848046,0.72417337],[0.09496737,0.8673003,0.26757553]],[[0.22932097,0.06117985,0.05550867],[0.5717687,-0.01583144,-1.0362136],[0.3951639,-0.00184457,-0.5246601]],[[0.3925199,-0.11556961,0.44707134],[-0.22137038,-0.10718393,0.45674607],[0.4878756,0.4408315,1.23722]],[[0.4083306,0.63318866,0.97525144],[0.03032762,-0.09426521,0.356666],[-0.25463045,-0.15376955,0.3624825]],[[0.03022995,-0.10600683,-0.23979971],[-0.40244904,0.4560807,-0.08578143],[0.25659114,-0.40532002,-0.36777413]],[[0.73376924,-0.7713153,0.18079454],[0.3919423,0.4886078,0.15907823],[-0.21508291,-0.48302248,-0.9217102]],[[0.22567414,0.39899737,-0.0492724],[-0.41132972,-0.55378145,-0.5472504],[-0.612572,-0.8128335,0.02128848]],[[-0.45840946,-0.04015734,0.4443072],[-0.3365265,0.42226917,-0.4748475],[0.38947633,0.31561702,-0.41365907]],[[0.32614774,-0.10619412,0.3700523],[0.33852038,0.13116945,0.39110723],[-0.23411527,-0.26759315,0.20967674]],[[0.46362722,0.00831244,0.7022598],[0.5439188,0.441105,0.722784],[0.528024,-0.24808456,0.21554995]],[[1.035305,-0.18724678,0.26284775],[0.05770473,-0.75603414,0.3613272],[0.7066588,-0.06521104,-0.24914268]],[[0.28170842,-0.29997557,0.07504286],[0.1443304,-0.15426914,-0.12196981],[0.8337606,0.6804418,0.45052058]],[[-0.4078168,0.02934877,-0.21222557],[-0.5577593,0.5529103,0.08389346],[-0.10711081,-0.41180772,0.15970926]],[[0.21671006,0.40400764,-0.6021877],[0.7179042,0.17144479,0.7101578],[0.37869975,0.19624098,0.06046851]],[[-0.7099632,-0.6539874,0.21733487],[-0.1203554,0.4708851,0.26448494],[-0.9975605,-0.00575296,-0.84142864]]],[[[1.29557610e-01,-4.71259892e-01,1.00122415e-01],[2.56476879e-01,-5.83498299e-01,3.73993039e-01],[2.88760543e-01,2.08454698e-01,-9.20166552e-01]],[[4.24182266e-01,-4.72254038e-01,-3.75491619e-01],[4.09691662e-01,2.22430900e-01,5.27472973e-01],[4.18300599e-01,1.32043973e-01,-3.88774723e-02]],[[-3.14628035e-01,4.80022937e-01,7.81356335e-01],[2.81812429e-01,2.33387481e-02,2.20928594e-01],[3.16137671e-01,-3.25374231e-02,2.45492116e-01]],[[6.25196576e-01,8.34840164e-02,-2.44920421e-02],[-6.46943808e-01,4.90783125e-01,6.59066319e-01],[4.01755959e-01,-5.55030763e-01,3.41778219e-01]],[[4.06275362e-01,-4.37972248e-01,3.06763560e-01],[2.91793942e-01,-3.81606460e-01,1.63489306e+00],[1.76455647e-01,-6.13247529e-02,9.86301422e-01]],[[-3.63158196e-01,1.66024536e-01,6.23714924e-01],[-1.09414741e-01,-9.90642756e-02,1.42022535e-01],[7.25162849e-02,2.27123141e-01,1.45963264e+00]],[[4.70924497e-01,-1.68108698e-02,4.86307412e-01],[1.67440012e-01,-5.24159551e-01,1.14557728e-01],[1.41204149e-01,-2.93244630e-01,-4.29602136e-04]],[[-7.54996419e-01,-2.75906682e-01,-1.17299467e-01],[1.23729274e-01,-2.43924007e-01,7.01609720e-03],[-1.29967049e-01,-3.72519046e-01,2.84602344e-01]],[[-1.30214483e-01,2.97689140e-01,-1.35141790e-01],[6.61429018e-02,-2.58847833e-01,3.81672800e-01],[-5.99046350e-01,6.53992891e-01,3.97979110e-01]],[[5.30432202e-02,4.44589555e-01,2.22853959e-01],[4.52957422e-01,-7.89914846e-01,5.36925066e-03],[5.38866937e-01,-8.99820775e-02,4.42920178e-01]],[[4.86277729e-01,-2.84830719e-01,1.51353143e-02],[-1.12811863e-01,-7.73327798e-03,-1.33841679e-01],[8.35258007e-01,3.06434393e-01,6.83758318e-01]],[[7.30529666e-01,3.50481987e-01,3.30678463e-01],[4.51149344e-02,3.42693120e-01,5.56844939e-03],[-1.14081658e-01,-5.82822502e-01,1.19105674e-01]],[[-1.17106907e-01,-2.73371339e-01,-1.72594324e-01],[-2.15152740e-01,-3.56704921e-01,-2.70849437e-01],[5.53191565e-02,3.25683737e-04,-8.43868196e-01]],[[3.83913487e-01,-1.47504076e-01,-5.88020444e-01],[1.37456059e-01,-6.29330426e-02,4.81863171e-01],[6.92066610e-01,9.09211874e-01,-3.86286452e-02]],[[1.45382777e-01,-2.49817938e-01,-2.80131698e-01],[5.55145442e-01,-1.66313678e-01,-1.63254693e-01],[-1.53241768e-01,3.96463752e-01,-6.46476448e-01]],[[9.64962915e-02,-2.22166538e-01,9.59152915e-03],[7.34556988e-02,-1.68145016e-01,3.81577015e-01],[-2.03102410e-01,6.62409514e-02,8.24813545e-01]],[[5.34748435e-01,-1.48082405e-01,4.13559586e-01],[-3.51923294e-02,1.43711250e-02,3.73574018e-01],[8.40945363e-01,-3.28406930e-01,7.65321329e-02]],[[2.54824787e-01,3.16180646e-01,4.99742031e-01],[6.09098375e-01,-2.76565820e-01,-4.49882835e-01],[4.20903355e-01,-2.17270926e-01,2.10620210e-01]],[[-1.12529077e-01,6.38189018e-02,-2.52484381e-01],[2.70133883e-01,4.10376102e-01,5.14042117e-02],[-8.25835764e-01,1.22724779e-01,7.97555447e-01]],[[3.16814929e-01,6.19818032e-01,3.28850925e-01],[-9.29980949e-02,4.21822995e-01,-4.03531313e-01],[-5.85511446e-01,1.65529117e-01,1.40246212e-01]],[[5.63361585e-01,-2.94130892e-01,7.31608450e-01],[2.98367798e-01,-1.38009280e-01,-3.04348111e-01],[4.20748562e-01,-7.71218305e-03,4.30425912e-01]],[[-4.45055693e-01,-1.36423063e+00,2.20337495e-01],[2.38928318e-01,-3.08725744e-01,8.36663842e-01],[6.11352921e-02,9.08460200e-01,4.82690901e-01]],[[-3.67221713e-01,7.87660778e-02,-1.65753114e+00],[2.96114892e-01,2.60335386e-01,-1.15335047e+00],[5.08596562e-02,5.81373930e-01,-1.08604109e+00]],[[1.74758598e-01,1.13900170e-01,3.22975993e-01],[4.71522063e-01,4.00347501e-01,2.90532053e-01],[6.34841681e-01,4.71943766e-01,-1.93781164e-02]],[[1.08381838e-01,1.87168166e-01,-6.16098166e-01],[3.07908326e-01,-2.85755783e-01,-1.08317412e-01],[3.51277888e-01,-1.70685679e-01,3.01298350e-01]],[[2.99884081e-01,-8.64959806e-02,-7.04900399e-02],[-4.37255830e-01,1.88318059e-01,-1.84435278e-01],[-1.04971394e-01,7.26270437e-01,3.21859986e-01]],[[-4.14840691e-02,6.47159517e-01,-4.57612783e-01],[-2.30099469e-01,4.35544491e-01,-1.57186761e-01],[-5.37700057e-01,1.14768259e-01,6.73637912e-02]],[[2.15507045e-01,-2.61342853e-01,4.18309309e-02],[-8.57667327e-02,5.77729166e-01,-3.78298104e-01],[-1.24074765e-01,1.15757689e-01,-5.32401145e-01]],[[5.42879760e-01,1.89892780e-02,1.82152808e+00],[2.69424558e-01,2.57431090e-01,5.04911721e-01],[-5.79362661e-02,-2.46963277e-01,4.86340284e-01]],[[-4.35356647e-01,1.33395106e-01,1.50833249e-01],[-6.28050208e-01,2.45987624e-01,4.49832350e-01],[-1.20454483e-01,4.10273187e-02,-4.70460765e-02]],[[-1.87053427e-01,-2.50316054e-01,-2.08667532e-01],[7.78261721e-01,-1.79329574e-01,-7.21891373e-02],[-5.26042700e-01,-2.07012117e-01,4.53397781e-01]],[[3.36617082e-01,4.14660454e-01,3.31138521e-01],[1.25496477e-01,3.69697601e-01,8.74611661e-02],[1.99905723e-01,-2.43964598e-01,3.33514273e-01]],[[-1.33883402e-01,-3.75560112e-02,-2.03496888e-01],[-2.20764294e-01,2.16044001e-02,5.28338909e-01],[1.01982355e-01,4.73108768e-01,-5.75984716e-01]],[[5.60004115e-01,-9.89870787e-01,4.45124298e-01],[1.00613141e+00,6.80360794e-01,1.02096736e+00],[5.45554161e-01,-6.35433912e-01,5.43236494e-01]],[[-3.05940700e-03,-6.07619062e-02,1.88365802e-02],[4.24453050e-01,-6.84821457e-02,8.36646497e-01],[-1.35117978e-01,5.26124120e-01,3.99737448e-01]],[[-1.90661609e-01,-7.95197964e-01,3.35732669e-01],[-8.22632313e-02,3.12452853e-01,-1.36394545e-01],[-1.41340062e-01,5.44460118e-01,1.66951120e-01]],[[-2.41435662e-01,-5.31883724e-02,1.95623204e-01],[1.51154429e-01,9.73867714e-01,1.11757195e+00],[1.05254436e+00,5.04180603e-02,-3.29984426e-02]],[[3.20275992e-01,1.07756920e-01,1.63139194e-01],[6.37546003e-01,-1.08467305e+00,2.07750961e-01],[4.61941719e-01,5.01642942e-01,9.80379879e-01]],[[4.87379372e-01,2.48899013e-02,-6.82169557e-01],[-4.46284890e-01,-4.52444181e-02,-5.47594070e-01],[2.42600986e-03,5.99863291e-01,6.00733101e-01]],[[3.27277809e-01,-7.16556013e-02,-3.84892464e-01],[1.64646387e-01,6.71838701e-01,5.36943138e-01],[-6.69037700e-01,3.94739032e-01,9.88423899e-02]]],[[[-7.34664738e-01,1.26864791e+00,3.24240237e-01],[-4.18300241e-01,-3.94774020e-01,1.22770138e-01],[-2.65755236e-01,-3.64320725e-02,-1.18104078e-01]],[[4.02897149e-01,-7.88998365e-01,-2.85241187e-01],[-2.37776399e-01,-4.62019175e-01,3.59064862e-02],[1.03686583e+00,7.91310370e-01,8.40233326e-01]],[[-1.84964865e-01,-7.54467130e-01,9.19108570e-01],[-3.60941321e-01,1.62230626e-01,3.19511503e-01],[-5.70373498e-02,-5.99172525e-02,-3.59431356e-01]],[[5.31218052e-01,2.91962102e-02,-1.30217865e-01],[-5.29759884e-01,-8.45624506e-01,-4.01465833e-01],[-9.50883269e-01,6.77541196e-02,-4.32346970e-01]],[[-1.89453885e-01,-3.90861362e-01,1.17953323e-01],[-1.40728921e-01,-3.14531028e-01,-6.81207895e-01],[-3.42047334e-01,-1.77440524e-01,-3.84485394e-01]],[[-2.67590106e-01,-1.04618222e-01,2.56229967e-01],[2.70970851e-01,-5.23141086e-01,-8.13766003e-01],[-2.12515563e-01,-7.17584968e-01,9.68584955e-01]],[[3.11085463e-01,-1.21227905e-01,7.71815360e-01],[-5.41779339e-01,-2.04550281e-01,1.67051271e-01],[3.65274161e-01,-2.43958935e-01,-3.66775572e-01]],[[5.21887720e-01,4.33153026e-02,-1.52720943e-01],[-2.93065995e-01,1.81600794e-01,5.49372017e-01],[4.69111383e-01,-2.88985651e-02,-6.07878268e-01]],[[5.83205283e-01,-3.10458355e-02,-3.40982378e-01],[3.18262905e-01,1.83029115e-01,-3.10557008e-01],[1.71040818e-02,9.84744914e-03,-6.75673008e-01]],[[1.06418288e+00,-1.86027154e-01,-1.00904834e+00],[-1.43393323e-01,6.66673034e-02,-2.40372613e-01],[5.65652907e-01,-5.96064217e-02,-9.62979317e-01]],[[4.37305033e-01,4.62788522e-01,-3.02560508e-01],[-1.70296878e-02,6.56361341e-01,-6.92655265e-01],[2.47129306e-01,-3.78513247e-01,3.33489507e-01]],[[-1.06484163e+00,-1.78649407e-02,3.13750178e-01],[4.43634510e-01,1.70356452e-01,1.08745378e-02],[-9.29886520e-01,1.23973221e-01,2.22575873e-01]],[[-6.98750138e-01,3.69320363e-01,1.67551830e-01],[2.96676397e-01,-2.66075023e-02,2.17975043e-02],[5.82198441e-01,2.69182801e-01,-1.81697428e-01]],[[6.73785090e-01,-3.60972732e-01,2.50740796e-02],[-7.23895803e-02,-5.06296635e-01,-8.20967332e-02],[-9.91865098e-01,5.35585761e-01,-7.09618479e-02]],[[-1.11189747e+00,6.75887048e-01,-1.42409233e-02],[-8.52761939e-02,5.38679838e-01,2.03147188e-01],[-2.57629454e-01,-2.90165722e-01,4.17839795e-01]],[[7.09586442e-01,5.65397181e-02,-6.97901964e-01],[-1.05274133e-01,3.01044118e-02,-3.84692281e-01],[-3.31543237e-01,-8.03978801e-01,-1.22008219e-01]],[[-1.31176440e-02,-2.07291707e-01,-2.48007432e-01],[7.70384222e-02,-4.56496119e-01,3.09188426e-01],[8.02794874e-01,2.67082989e-01,4.32646811e-01]],[[-4.52687681e-01,7.31234074e-01,-4.07373965e-01],[1.27450153e-01,2.13944659e-01,2.65021712e-01],[-6.63228035e-01,1.18786879e-01,3.34779084e-01]],[[-3.32246810e-01,-2.30164558e-01,-8.09537172e-02],[-6.50308058e-02,-6.10550106e-01,-5.45532644e-01],[-8.87559354e-01,3.06965336e-02,-4.30577882e-02]],[[8.30302298e-01,-6.11957163e-02,5.23590326e-01],[-1.26284286e-01,1.68117240e-01,-1.01636291e-01],[-4.59695071e-01,2.65285559e-02,4.87383038e-01]],[[4.50889498e-01,-7.70784467e-02,-1.17476255e-01],[-5.26612937e-01,4.78012860e-01,-2.36552715e-01],[-2.37589944e-02,2.11986974e-01,1.22485161e-01]],[[-5.27120948e-01,8.87005866e-01,4.77365017e-01],[-3.00913841e-01,-4.21507865e-01,1.08308583e-01],[-2.90296257e-01,5.18697381e-01,1.89201996e-01]],[[2.71794319e-01,-2.56694376e-01,8.78217280e-01],[1.66953355e-01,3.86292905e-01,-1.74991950e-01],[1.40806988e-01,-2.33801931e-01,4.10748303e-01]],[[-8.39876473e-01,3.52165073e-01,4.45243478e-01],[-2.60686427e-01,2.52735257e-01,-1.87741861e-01],[6.32829666e-01,-2.16540024e-01,-6.39951050e-01]],[[1.02078664e+00,-4.16854560e-01,-4.55080539e-01],[-3.24944735e-01,5.34988642e-01,3.48234415e-01],[-2.55181611e-01,2.08567590e-01,8.72953475e-01]],[[-1.52971402e-01,-2.66236603e-01,1.00197420e-01],[-5.81869185e-01,1.88311562e-01,-1.24454446e-01],[4.19316143e-02,-9.89406884e-01,1.78162962e-01]],[[-1.50821924e-01,9.23156440e-01,-3.26945007e-01],[-1.00287509e+00,-2.23927200e-01,3.26376140e-01],[5.97313046e-02,4.21293676e-01,-6.11454487e-01]],[[-1.13521516e-01,4.29511547e-01,-1.72018126e-01],[5.13182953e-02,-4.13787574e-01,1.27422258e-01],[-4.78626221e-01,-2.78196990e-01,3.77127618e-01]],[[-5.23770511e-01,1.60634845e-01,-1.93778872e-01],[-2.79481769e-01,3.46700758e-01,-1.28147244e-01],[2.57408470e-01,3.12655747e-01,-9.72265378e-02]],[[-4.24201220e-01,-5.89916587e-01,3.66088480e-01],[2.47779921e-01,-3.09963167e-01,4.82104011e-02],[2.90067673e-01,1.46451220e-01,3.91963452e-01]],[[2.28753996e+00,-4.03790116e-01,-1.01353903e-03],[1.84678912e+00,-2.06141993e-01,-2.04028010e-01],[3.22259843e-01,-4.87892896e-01,4.33871858e-02]],[[4.83690808e-03,-2.02853367e-01,6.60986826e-02],[-2.51218826e-01,-2.82345772e-01,4.07129288e-01],[-4.35961902e-01,1.95407480e-01,1.03123736e+00]],[[9.39481914e-01,-4.95965064e-01,-1.27886562e-02],[7.49555230e-01,-2.88025022e-01,1.18498534e-01],[2.12409243e-01,-2.10150108e-01,1.43458128e-01]],[[1.38793066e-01,3.20819736e-01,-5.50725162e-01],[4.20147717e-01,6.04207158e-01,-1.81841508e-01],[-8.59508276e-01,-1.50940686e-01,-7.58311152e-01]],[[-3.15608531e-01,4.76281285e-01,1.67771384e-01],[-3.10934305e-01,-1.27502993e-01,-3.81502658e-02],[1.34279821e-02,-8.75296235e-01,-2.17768952e-01]],[[-6.05589330e-01,2.71416932e-01,-3.89031470e-01],[2.61178821e-01,6.51182681e-02,1.79401070e-01],[9.40498054e-01,-6.02463558e-02,-6.00491166e-01]],[[-5.87098837e-01,-1.53889865e-01,2.80472994e-01],[4.91269201e-01,-1.95644423e-01,-2.44408160e-01],[6.74575567e-01,8.13026488e-01,2.77729988e-01]],[[-5.28026819e-01,7.14561418e-02,4.36597347e-01],[3.37567449e-01,-4.52556908e-01,4.43932921e-01],[-4.96535748e-01,-5.04359782e-01,1.00348607e-01]],[[2.54474849e-01,-2.80698121e-01,6.80653691e-01],[-5.48379421e-01,5.28788149e-01,1.12225972e-02],[-4.95863184e-02,-2.21940488e-01,-3.22433084e-01]],[[4.38146621e-01,-1.14806674e-01,-4.78238165e-01],[-7.79669732e-02,1.92703888e-01,-5.49756944e-01],[5.34435153e-01,-6.75039649e-01,-1.03441107e+00]]],[[[-4.76842403e-01,-2.05938503e-01,2.44073585e-01],[-1.58695206e-01,8.45622659e-01,3.44355315e-01],[-3.39847773e-01,3.05365831e-01,1.99332863e-01]],[[5.65386117e-02,5.95966354e-02,3.40058506e-02],[3.80483754e-02,4.68259901e-01,-2.78225988e-01],[-1.13645589e+00,-3.26335013e-01,-1.10281050e+00]],[[-7.44232535e-01,-4.23863046e-02,7.04746068e-01],[-2.13226378e-01,1.42902806e-01,1.53812453e-01],[3.86583060e-01,-1.12143122e-01,1.04060113e+00]],[[-3.42692174e-02,4.85374361e-01,6.28132820e-01],[-1.17475264e-01,-1.69671997e-01,5.77743888e-01],[1.81717128e-01,3.29377204e-01,-9.61857364e-02]],[[-7.77859017e-02,9.25964117e-02,-8.34556401e-01],[-1.68234080e-01,-4.02463645e-01,-2.51232117e-01],[-4.26996291e-01,-2.41112858e-01,8.40498865e-01]],[[-4.89435375e-01,6.88706189e-02,-7.90211558e-02],[4.56110798e-02,4.41146493e-01,2.14294910e-01],[-4.54071909e-01,-8.08207020e-02,-1.12713508e-01]],[[1.36287779e-01,-4.62767966e-02,-4.26658332e-01],[-1.08089173e+00,6.02963984e-01,-1.24214664e-01],[7.90778473e-02,-7.56720066e-01,-7.50303805e-01]],[[4.53873314e-02,7.10040450e-01,1.07326083e-01],[6.59665048e-01,-3.58808547e-01,2.06180215e-01],[4.14349765e-01,2.69314855e-01,2.06057593e-01]],[[-1.40505672e-01,1.58401117e-01,1.71437208e-02],[2.26783380e-01,-3.11329901e-01,-3.34823847e-01],[-2.31180295e-01,-6.62515163e-01,-7.68808246e-01]],[[-3.33665997e-01,-2.30934337e-01,2.64323145e-01],[9.68693912e-01,-9.91117299e-01,9.05967832e-01],[1.90087467e-01,-4.90566976e-02,-7.99638927e-01]],[[2.25681532e-02,5.48031986e-01,-5.39118826e-01],[5.40143490e-01,-8.69712412e-01,-6.94469810e-01],[3.33507150e-01,6.55529082e-01,-7.62090266e-01]],[[4.57876250e-02,3.62992823e-01,1.36440154e-02],[1.23618841e-02,4.13546890e-01,3.23107153e-01],[-3.48208874e-01,2.57763872e-03,-3.34715843e-02]],[[-1.96970433e-01,9.03230369e-01,-1.86327815e-01],[4.38342802e-02,2.41360426e-01,-1.33008212e-01],[3.35255742e-01,2.44481444e-01,-1.28226388e+00]],[[-7.59079913e-03,-3.80734801e-01,2.04146340e-01],[-6.38038099e-01,1.08650351e+00,9.08169150e-02],[-7.66666412e-01,4.08045501e-02,4.80546430e-02]],[[6.28238916e-03,3.39788735e-01,-5.91343939e-01],[-2.69389302e-01,-5.77770054e-01,-1.46992981e-01],[8.93576384e-01,-1.75080925e-01,2.61103511e-01]],[[-1.27444327e-01,1.70848593e-01,-1.69356394e+00],[5.66823065e-01,3.63993138e-01,-5.70660710e-01],[-7.24196076e-01,5.03333919e-02,-1.02408195e+00]],[[2.26716861e-01,-1.33376062e-01,-2.78185785e-01],[-3.39695951e-04,-1.73849419e-01,-2.21255586e-01],[4.54944879e-01,-3.81764248e-02,9.64187562e-01]],[[2.27466017e-01,1.14548445e+00,-6.75562978e-01],[-1.22681998e-01,-5.65141499e-01,-1.61541343e-01],[7.43227124e-01,-6.77074492e-01,-9.72571850e-01]],[[8.49617004e-01,-1.72677428e-01,5.43644369e-01],[3.69409591e-01,5.44221938e-01,-5.98991394e-01],[-1.77643865e-01,-3.77548039e-01,1.18652776e-01]],[[-1.74744770e-01,-7.68222436e-02,-6.60942912e-01],[-1.02221943e-01,-1.79701477e-01,-9.26688090e-02],[1.28435463e-01,5.60998261e-01,-6.09124564e-02]],[[-2.25144103e-01,-2.62884468e-01,6.66483492e-02],[-3.31413537e-01,2.13110343e-01,-7.32453242e-02],[2.81342745e-01,-3.72266620e-01,-6.03653491e-02]],[[3.34667832e-01,-5.39844930e-02,8.20786357e-01],[3.59944224e-01,9.53679010e-02,-2.52294719e-01],[2.56483197e-01,-2.64220834e-01,-1.46486416e-01]],[[9.26500112e-02,6.57378078e-01,-4.89364207e-01],[-1.23332411e-01,6.07283950e-01,8.68238043e-04],[3.12933743e-01,-2.19946727e-01,4.80220258e-01]],[[7.24570081e-02,6.48354888e-01,-1.16732597e+00],[2.06485912e-01,2.69139200e-01,2.23291412e-01],[-4.87001628e-01,-3.73236209e-01,5.04212499e-01]],[[2.52947688e-01,-3.71153094e-02,-3.84530455e-01],[5.32252133e-01,-1.45641312e-01,-1.08899385e-01],[-2.32032344e-01,-7.73265306e-03,-1.08495384e-01]],[[-5.22157609e-01,-3.28818828e-01,-2.46094808e-01],[-1.14613377e-01,3.35687213e-02,4.52977151e-01],[2.78282642e-01,-3.17411482e-01,-6.00080848e-01]],[[-6.44695640e-01,7.33799338e-02,3.56619060e-02],[-3.05696249e-01,4.15463358e-01,-9.60587680e-01],[2.08508611e-01,6.21818542e-01,1.73768163e-01]],[[-9.46372598e-02,-7.37048462e-02,-6.11108840e-01],[-5.15806258e-01,5.27871788e-01,-6.46719277e-01],[3.17418218e-01,-7.76305437e-01,-1.44166976e-01]],[[-4.05325174e-01,4.02268231e-01,-2.06826165e-01],[3.89064640e-01,1.88804328e-01,1.67842835e-01],[3.73332560e-01,1.11047804e-01,-1.52746081e-01]],[[-6.47334993e-01,-2.64559150e-01,-9.86132920e-01],[1.39293820e-01,-4.95412022e-01,-2.60335002e-02],[-5.60371041e-01,5.11644721e-01,8.90076086e-02]],[[-9.22505558e-02,-4.00010258e-01,6.34388685e-01],[1.44867286e-01,5.23456395e-01,4.89235409e-02],[-2.29704916e-01,-3.94600958e-01,2.42481697e-02]],[[2.00146865e-02,-7.48856723e-01,2.67707139e-01],[1.28594011e-01,-2.49936819e-01,8.30302238e-02],[-7.21406937e-02,1.45746050e-02,-2.89074890e-02]],[[3.72550935e-01,2.03237727e-01,-5.78817248e-01],[-1.12666763e-01,-3.96320790e-01,-6.36648834e-02],[-2.00149968e-01,6.46086216e-01,7.31057674e-02]],[[4.00279582e-01,-3.22199792e-01,-6.04886949e-01],[3.63502383e-01,-4.43357348e-01,-5.17892838e-01],[3.56642842e-01,3.48893911e-01,5.78994490e-02]],[[-4.47233945e-01,-2.25681320e-01,-3.85599047e-01],[-8.57732669e-02,-8.07619929e-01,3.41451883e-01],[-3.97043452e-02,-4.43529278e-01,-5.11279047e-01]],[[1.20418930e+00,-4.06587869e-01,-3.25482249e-01],[7.87070930e-01,1.40790129e-02,2.66586691e-01],[2.51849070e-02,3.81367058e-01,-3.68712395e-01]],[[-7.33275041e-02,-9.39792454e-01,-4.42357033e-01],[-6.54389143e-01,4.07882631e-01,-1.29387692e-01],[-7.88948163e-02,-4.38674718e-01,2.80410320e-01]],[[2.00360090e-01,1.11172847e-01,-9.01648700e-01],[5.54016829e-01,-2.19334081e-01,-2.59185582e-01],[8.81457508e-01,4.54274416e-01,7.72307456e-01]],[[-3.74808162e-01,-5.62234148e-02,2.57774234e-01],[-9.80528355e-01,3.02017462e-02,5.26850857e-02],[-2.29028985e-01,3.98713261e-01,7.40741253e-01]],[[8.24279428e-01,-2.26111442e-01,-1.37468830e-01],[-1.06899835e-01,-2.17986703e-01,-9.96319354e-02],[3.29199284e-01,-4.09121662e-01,-4.42636609e-02]]],[[[-9.71097946e-01,-5.49838066e-01,-1.30283739e-02],[-4.82166350e-01,-3.09907440e-02,-1.10542916e-01],[-4.59578723e-01,-5.75048804e-01,7.22564101e-01]],[[7.15417564e-01,5.45715690e-02,4.66394991e-01],[5.50754666e-01,2.17899993e-01,-4.13039178e-02],[-6.44612461e-02,-1.00768521e-01,2.61072725e-01]],[[-2.91985810e-01,3.41807872e-01,1.28460869e-01],[1.33670315e-01,-1.12975528e-02,4.86213863e-01],[-1.47792390e-02,6.63952172e-01,-1.15785033e-01]],[[7.43152857e-01,3.86307746e-01,4.44445044e-01],[-7.21813321e-01,-3.75068039e-01,4.21772420e-01],[-6.54204488e-01,1.18928008e-01,9.83671024e-02]],[[-9.98414040e-01,-1.64845049e-01,-5.37314355e-01],[-3.25838119e-01,-4.99581471e-02,-1.13334060e+00],[-9.18635011e-01,2.18268260e-01,-8.96143734e-01]],[[-2.42208302e-01,-3.46350521e-01,4.44459647e-01],[1.72521383e-01,6.02698803e-01,-4.12568539e-01],[1.62558988e-01,-6.73093200e-02,4.26576227e-01]],[[3.85431528e-01,-1.57508284e-01,9.34116095e-02],[3.38694066e-01,-2.32876670e-02,-2.32388824e-01],[3.87612015e-01,-4.69418205e-02,-5.00477493e-01]],[[2.87950551e-03,5.27257740e-01,-6.34646863e-02],[2.14052022e-01,-5.20884022e-02,6.08232394e-02],[1.89994857e-01,4.25075680e-01,-4.06787366e-01]],[[-1.15291722e-01,1.06266346e-02,7.20784307e-01],[-2.50153672e-02,5.55652142e-01,-7.61578307e-02],[2.43968979e-01,5.43045640e-01,6.48615003e-01]],[[-6.56364560e-01,3.21173877e-01,1.93872139e-01],[1.74257293e-01,-7.63148248e-01,-1.35813564e-01],[1.14096797e+00,3.03818196e-01,5.20105660e-01]],[[-1.19897790e-01,-2.32432127e-01,-1.66310951e-01],[3.59570682e-01,1.83735415e-01,-3.69902045e-01],[-2.48714864e-01,4.33995903e-01,-2.28589490e-01]],[[-3.21591139e-01,1.97708309e-01,-5.81404924e-01],[-2.01442968e-02,-2.20901161e-01,4.72031355e-01],[3.94675553e-01,-4.08436321e-02,-2.12931842e-01]],[[4.64298636e-01,-6.08550310e-01,-5.68933308e-01],[8.93603936e-02,1.92500502e-01,-1.73724145e-01],[-4.60590035e-01,-1.66447416e-01,-1.01316345e+00]],[[1.05080508e-01,-5.91858506e-01,6.27419949e-01],[-8.10000241e-01,-7.45654345e-01,1.55735329e-01],[-9.32076633e-01,3.39543939e-01,1.90793410e-01]],[[-6.74090683e-01,-4.77073967e-01,2.95798659e-01],[-6.48490429e-01,-1.58338204e-01,4.20430377e-02],[-3.39152396e-01,-3.08882773e-01,-2.62043297e-01]],[[-7.32131422e-01,3.20853531e-01,-2.04828203e-01],[4.89173532e-02,-9.69716191e-01,-8.40825200e-01],[-2.10483313e-01,-5.87892771e-01,-6.46937370e-01]],[[1.97950467e-01,3.23795341e-02,5.14057539e-02],[6.40532315e-01,-2.83987164e-01,-6.81414530e-02],[1.24033213e+00,3.48811030e-01,-2.25819126e-01]],[[1.78294718e-01,-3.45296949e-01,3.89313959e-02],[1.05530739e-01,5.38866699e-01,-4.59820777e-01],[7.68340006e-02,4.32387173e-01,4.06171381e-02]],[[3.50342453e-01,-1.61661670e-01,2.57579936e-03],[5.73662221e-01,-4.09967154e-02,6.74812019e-01],[2.75100499e-01,-3.57164651e-01,6.69494033e-01]],[[-3.06245923e-01,-4.17346179e-01,3.31026584e-01],[3.17324042e-01,-7.63980269e-01,3.81371886e-01],[8.00255001e-01,1.08461671e-01,9.53624606e-01]],[[-5.24724960e-01,-1.88032553e-01,-7.34327257e-01],[-6.30176365e-01,2.01849580e-01,1.90125853e-01],[-1.11791044e-01,-4.30382155e-02,2.62616396e-01]],[[-4.27418023e-01,3.74805629e-01,-2.06805348e-01],[2.27402121e-01,-2.97232103e-02,-4.19499397e-01],[3.02033722e-01,-2.17264369e-01,-5.19809306e-01]],[[6.74979985e-01,4.31957483e-01,1.21178257e+00],[-4.62874055e-01,7.17186153e-01,8.91999662e-01],[-2.22210050e-01,1.93282485e-01,7.15537429e-01]],[[3.91530216e-01,-7.93710053e-01,-1.96637914e-01],[-2.13132992e-01,-3.11477706e-02,3.15906554e-01],[-5.87633789e-01,-7.02207804e-01,2.66974390e-01]],[[8.25516507e-02,-2.56386638e-01,5.96177876e-01],[6.01616859e-01,-2.61838168e-01,2.17927188e-01],[2.98775464e-01,-1.92726403e-01,1.11171091e+00]],[[-3.33180636e-01,2.60423452e-01,-4.23471361e-01],[3.01167548e-01,8.85656774e-02,1.87545061e-01],[6.80332184e-02,-7.92947263e-02,-2.11187094e-01]],[[-4.65800136e-01,-2.18774289e-01,2.51186162e-01],[-3.38673979e-01,4.22370672e-01,4.76476431e-01],[-1.78766295e-01,-1.20020941e-01,9.60046113e-01]],[[-6.68975234e-01,-2.73240864e-01,-2.84661353e-03],[3.32879484e-01,3.89795423e-01,1.22308642e-01],[1.24917758e+00,2.27618963e-01,5.22246957e-01]],[[-5.77049077e-01,-6.30298197e-01,-1.05318785e+00],[-2.78838098e-01,-5.88974841e-02,1.25078723e-01],[5.57092309e-01,5.21597676e-02,-4.48147893e-01]],[[1.35710165e-01,-1.20914422e-01,-4.13792133e-01],[4.36147526e-02,-7.20043033e-02,1.97469741e-02],[-7.74751961e-01,6.02716386e-01,-4.92242575e-01]],[[-1.90581493e-02,-6.44712508e-01,-7.87011266e-01],[-5.18153906e-01,-3.20095569e-01,-1.64510831e-01],[8.21490511e-02,-6.60471559e-01,9.14265141e-02]],[[-7.09959149e-01,-1.83726817e-01,-3.81191820e-01],[7.89619327e-01,-1.01421073e-01,-2.11660743e-01],[-4.30285484e-01,4.69662100e-01,-3.88799071e-01]],[[1.40655681e-01,-3.72521132e-01,3.51337284e-01],[1.69316694e-01,1.15368277e-01,3.96352530e-01],[-1.92072108e-01,-1.76005766e-01,6.75346375e-01]],[[1.52610749e-01,-1.15131333e-01,9.04478878e-03],[-5.14525414e-01,-4.24672872e-01,-4.03747112e-01],[-2.64861524e-01,1.04075938e-01,-9.09687996e-01]],[[-2.63378829e-01,4.51233596e-01,-4.07286674e-01],[-7.02616990e-01,5.29862382e-02,-9.75898981e-01],[-8.28950226e-01,1.08669223e-02,-2.92240053e-01]],[[-8.04007202e-02,7.29119927e-02,-2.19981633e-02],[-5.08529663e-01,-2.21439287e-01,-7.38992393e-02],[-7.27599442e-01,-4.26309973e-01,6.97596490e-01]],[[8.04057717e-02,2.40252927e-01,-1.36225075e-01],[1.41017109e-01,-4.43807915e-02,-1.56467948e-02],[-6.40284300e-01,-5.24495661e-01,2.80965030e-01]],[[-6.41175091e-01,3.13342094e-01,1.68406174e-01],[4.00819838e-01,-3.15044373e-01,3.70041430e-01],[4.34432507e-01,7.55445242e-01,-6.27096668e-02]],[[-7.25709081e-01,-3.52706313e-01,2.98487931e-01],[-1.92729669e-04,5.70160076e-02,1.88418940e-01],[4.51299362e-02,-2.21785400e-02,-2.27180347e-01]],[[-3.74768406e-01,-3.19001198e-01,8.41565907e-01],[-1.60334677e-01,2.23014563e-01,-1.04550278e+00],[5.92516840e-01,-5.75571775e-01,3.30434710e-01]]],[[[-0.2635146,-0.39103675,-0.2248611],[-0.49077356,0.10729635,-0.12351685],[-0.10568003,-0.19967093,-0.21249212]],[[-0.5363205,0.4671859,-0.03179128],[0.173013,-0.34868366,0.10970557],[-1.0556959,-0.23214473,-0.01986276]],[[-0.00157619,-0.6983225,0.6098233],[0.07448081,0.14196138,0.18910807],[-0.8134909,0.01431892,-0.40396297]],[[0.4871801,-0.43052506,0.02976174],[0.12209253,0.05133892,0.31671548],[0.24842924,-0.676004,-0.17965612]],[[0.21071558,0.11776473,-0.4661344],[-0.33716193,-0.27004468,0.44936353],[0.08212751,-0.586633,0.17915414]],[[0.33780122,-0.0274092,-0.313752],[1.0890568,-0.2900753,-0.6846928],[-0.34364522,0.15523875,-1.1478459]],[[-0.4378948,0.2713646,0.504909],[-0.45726895,-0.22908446,-0.0243866],[-0.2705679,0.00824447,-0.950835]],[[-0.32458156,0.20365034,-0.18658546],[-0.10356532,0.23184648,-0.22323039],[-0.13439955,0.01849749,0.45679528]],[[-0.05498398,-0.03257344,-0.6836889],[0.26523304,0.15262735,-0.7207899],[-0.83100045,-1.0290893,-0.05657548]],[[-0.97008115,-0.36962494,-0.5122326],[0.04205621,-0.27713066,-0.6888974],[-0.47166616,0.3172922,-0.51201874]],[[0.34824982,-0.61357236,0.01061217],[-0.41456524,0.03585872,0.45802253],[0.17878234,-0.11761355,-0.51019007]],[[0.30771413,-0.5832077,-0.49751285],[0.31796753,-0.10621586,0.29109284],[1.2020463,-0.3013898,-0.04847728]],[[0.590568,0.12906615,0.54731417],[0.47171104,-1.250946,-0.17773594],[0.20939453,-0.24573584,-0.52711046]],[[0.2342814,0.46352375,-0.31043044],[0.9688702,-0.0669712,0.45197493],[0.32329783,-0.17707814,0.24555753]],[[0.05398346,0.3368194,0.09398467],[-0.5461566,0.48642388,0.24278498],[-0.648744,0.2451401,0.55979687]],[[-0.1214397,-0.44636762,-0.01082813],[0.10840303,-0.36123967,-0.1586018],[-0.0405883,-0.42852783,-0.05080912]],[[-0.07107157,0.5840379,-0.31301418],[-0.7093628,0.6698424,0.08710662],[-0.94219345,-0.21208775,0.37145373]],[[0.4665583,0.63647836,-0.24982],[0.7049679,0.31839257,-0.41590548],[-0.09546819,-0.42256385,0.01992119]],[[-0.12759393,-0.23537196,1.1178364],[0.37496057,-0.3420559,-0.3920065],[-0.19607455,0.3509465,-0.04699044]],[[-0.3871853,0.2086488,-0.2429859],[0.23617648,0.1798844,-0.4851686],[-0.64017457,-0.236952,-0.8288489]],[[-0.09131885,0.2538795,-0.27980903],[0.25444487,-0.7589687,-0.5539715],[0.29684064,0.39004594,0.15371585]],[[0.17174676,-0.18581566,-0.28225923],[-0.1501235,0.28804466,-0.06768157],[0.09884594,0.3067884,-0.5669216]],[[-0.28718874,-0.21290846,-1.0292934],[0.52320874,-0.10753496,-0.07382336],[0.44913638,0.18951263,-0.70141655]],[[-0.50079584,-0.5734113,0.70777947],[-0.23356813,0.33832195,-0.61846566],[0.37271228,0.29616156,-0.6093744]],[[-0.9417213,-0.19589062,-0.6203908],[-0.481484,0.25563508,0.18040125],[-0.6927277,-0.2591252,-0.40118566]],[[0.48069218,0.30218086,0.6477029],[0.90732896,0.04167882,-0.20656164],[0.42458284,-0.9877922,0.9150739]],[[0.2671156,0.00904671,-0.10063729],[-0.07435779,-0.40889198,-0.30171347],[0.29935786,-0.6057365,-0.89067197]],[[0.8768144,0.34851563,-0.42228374],[0.13605611,0.32749683,-0.35184708],[-0.13106194,0.1344106,-0.39134875]],[[0.60812527,-0.31200227,0.22975685],[-0.5036536,0.199696,0.55477566],[-0.29510894,-0.05161735,-0.1477918]],[[0.53356403,-0.17333049,0.22858602],[0.14266875,0.25870842,-0.55776346],[0.12087039,0.06937664,0.46609682]],[[-0.83060706,0.09224374,-0.02272831],[-0.60315645,0.08603951,-0.06463166],[-0.54827803,-0.7937802,0.17209777]],[[0.4936773,-0.29096502,0.17633711],[-0.3747976,0.23352908,-0.353672],[-0.47701275,-0.31775305,0.5152216]],[[-0.89842683,-0.6765403,-0.5436569],[-0.04529592,1.0418142,-0.4219006],[0.22153826,0.3897575,0.20119658]],[[0.5274436,0.07257436,-0.52511376],[0.06326215,0.10948745,0.08354405],[-0.18438043,-0.29336467,0.304915]],[[-0.18162358,-0.5525332,0.01209892],[0.23207934,0.77161205,-0.5789046],[0.4417023,0.05741839,0.27839997]],[[-0.51540494,0.04563064,0.86201257],[-0.36198515,-0.53226286,-0.25073087],[-0.26404798,0.32619107,0.11118061]],[[0.12105864,0.37819666,-0.01761827],[-0.71361977,0.6596687,1.2548229],[0.1020182,0.16021155,-0.17699008]],[[0.46535712,-0.09313658,0.21331011],[0.09020367,0.01601822,0.59239364],[0.5531902,0.22032665,-0.18096189]],[[0.99807405,0.67582023,0.9774362],[-0.42810696,0.18177254,0.2515966],[0.43240505,0.01401048,-0.33984268]],[[-0.7621096,-0.01696541,-1.0038006],[-0.9130363,0.04657211,0.20911191],[-0.13325888,-0.08755481,-0.31843248]]],[[[-4.67485934e-03,1.24208011e-01,1.12374358e-01],[-3.79061073e-01,-3.49146366e-01,2.25948200e-01],[2.28302732e-01,4.41531956e-01,3.13523635e-02]],[[-5.38007438e-01,-6.72687218e-02,-2.88367063e-01],[7.23687768e-01,-4.65275377e-01,1.20858774e-01],[3.84730786e-01,-4.72923130e-01,-1.38827249e-01]],[[1.10201305e-02,-1.76766858e-01,2.82524884e-01],[4.33940113e-01,2.51504391e-01,8.94079506e-01],[-2.20629826e-01,-4.19843763e-01,-3.62323225e-02]],[[5.57232738e-01,1.13776170e-01,3.10456902e-01],[-2.62206607e-02,-6.58652186e-01,1.27369791e-01],[8.36356878e-01,7.26708710e-01,4.73331630e-01]],[[3.47282708e-01,-3.51673454e-01,-4.24612686e-02],[-6.58001453e-02,4.63375121e-01,2.70944089e-01],[3.95399421e-01,-8.79582241e-02,6.84082508e-02]],[[-6.88061059e-01,2.13181004e-01,-1.71597958e-01],[4.57951814e-01,-6.78924739e-01,2.75541246e-01],[1.63322091e-01,-1.24650948e-01,-5.99722452e-02]],[[8.50137830e-01,1.89019904e-01,1.24839142e-01],[5.63687161e-02,-4.03584749e-01,4.95156944e-02],[1.40984333e+00,5.60838044e-01,5.60515106e-01]],[[-1.79069370e-01,-3.21767569e-01,-3.00436646e-01],[-5.54865366e-03,-2.45444581e-01,-9.12638128e-01],[1.42031515e+00,-7.93630362e-01,6.84900880e-01]],[[-4.77420896e-01,8.70043695e-01,2.12463379e-01],[-8.91171455e-01,-3.84924024e-01,-6.54119551e-02],[-2.40172029e-01,1.84762329e-01,-1.15969680e-01]],[[4.41956103e-01,2.30574911e-03,5.71126677e-03],[-4.44145918e-01,-4.09635633e-01,1.80612162e-01],[-5.30125797e-01,7.58404255e-01,-3.62487078e-01]],[[1.81043953e-01,-5.76076567e-01,-3.85058254e-01],[2.60080367e-01,-2.85838604e-01,1.30492598e-01],[7.75547028e-01,2.13927537e-01,-7.18947127e-02]],[[-1.53187919e+00,-6.80734366e-02,1.44589454e-01],[-1.33474082e-01,-1.64991274e-01,-9.93642658e-02],[-2.76268441e-02,4.25721481e-02,-4.28295374e-01]],[[1.44638389e-01,3.16818953e-01,-6.52312398e-01],[4.36021835e-02,1.30668625e-01,1.79033607e-01],[-2.43124351e-01,5.62361002e-01,7.64279589e-02]],[[9.01015043e-01,-8.59084785e-01,-2.64349967e-01],[4.89802539e-01,-1.02345097e+00,-4.05776709e-01],[1.29597378e+00,6.47762835e-01,3.16876531e-01]],[[-2.13132441e-01,-2.65199840e-01,-3.02443057e-01],[1.02506042e+00,5.69621503e-01,9.31973398e-01],[-9.00790393e-01,3.64901215e-01,2.70822942e-01]],[[6.84763849e-01,-5.05508808e-03,-1.73609611e-02],[2.25315809e-01,9.96241793e-02,-1.03236890e+00],[-4.60457876e-02,3.37242693e-01,-8.28324333e-02]],[[1.51900500e-01,-2.04923809e-01,1.94029585e-01],[-1.08315945e-01,4.85184014e-01,-1.73223093e-01],[1.39347926e-01,4.49366719e-01,7.93048561e-01]],[[-3.61786604e-01,1.15975507e-01,2.03964844e-01],[5.21717846e-01,3.84453684e-01,-8.48167419e-01],[2.47194126e-01,5.77621460e-01,2.30719388e-01]],[[2.70116776e-01,-2.27365732e-01,7.75259174e-03],[5.70129752e-01,-5.56556702e-01,1.98113009e-01],[3.47421378e-01,6.08636200e-01,2.84731746e-01]],[[-6.94075674e-02,2.63403058e-01,5.37393242e-03],[5.47860712e-02,1.97278649e-01,3.42657119e-01],[4.85027015e-01,1.25265986e-01,-2.75652930e-02]],[[5.61563356e-05,4.38954949e-01,4.00079712e-02],[-1.09695911e-01,1.99426755e-01,-4.46925730e-01],[8.28201473e-02,-3.45247000e-01,-3.82831134e-02]],[[-1.77229464e+00,1.75411448e-01,-6.20343029e-01],[-6.49282873e-01,3.24106961e-01,-4.18948352e-01],[-9.32964027e-01,-2.41955612e-02,2.61010081e-01]],[[-4.22663391e-02,5.22643745e-01,-3.71970177e-01],[3.62923265e-01,-2.11311147e-01,1.46247134e-01],[3.23153138e-01,-4.59614426e-01,-1.51090607e-01]],[[4.82174814e-01,2.63231471e-02,-2.72377562e-02],[2.16316029e-01,-6.87173128e-01,-1.93672210e-01],[-9.23979208e-02,5.82044683e-02,-3.18659127e-01]],[[-3.08028251e-01,2.80861050e-01,9.58921984e-02],[2.83477623e-02,-3.11750859e-01,-8.94934833e-02],[-2.41881669e-01,-1.71491206e-01,-7.15646818e-02]],[[3.88509333e-02,6.18688703e-01,5.40476680e-01],[-9.59239364e-01,-4.87926960e-01,-3.85597855e-01],[4.70442593e-01,5.67553699e-01,1.18511069e+00]],[[-4.17582430e-02,-2.51700938e-01,1.09573267e-01],[3.84946853e-01,-2.63182074e-01,-3.01037520e-01],[8.25467467e-01,1.53100580e-01,-3.17966454e-02]],[[-3.43984067e-01,-1.31947137e-02,3.07001948e-01],[6.52647495e-01,-1.38788372e-01,-1.79961413e-01],[-2.84616292e-01,-4.57775950e-01,-1.12475969e-01]],[[-4.71561372e-01,2.59320140e-01,-3.72791559e-01],[8.97949748e-03,-7.44875729e-01,-2.38176405e-01],[-8.48366413e-03,3.05277556e-01,-1.56191841e-01]],[[6.16704524e-01,2.68700153e-01,-6.41445100e-01],[-2.58682579e-01,4.97087799e-02,-4.91668552e-01],[-6.57419264e-01,-2.04881117e-01,4.01544988e-01]],[[2.10547829e+00,-5.80261588e-01,-2.29590669e-01],[9.15816665e-01,-7.58228421e-01,-4.37553460e-03],[9.69693601e-01,-1.89834505e-01,-4.63633358e-01]],[[-2.81097889e-01,-1.68122366e-01,4.35801148e-01],[-6.73497021e-01,-3.29782486e-01,7.24767268e-01],[2.37454712e-01,2.37164676e-01,4.82144713e-01]],[[1.38928986e+00,-2.18437582e-01,-2.86727049e-03],[-8.90991688e-02,1.10975496e-01,-3.04083824e-01],[3.90425831e-01,-3.28189321e-02,4.16689157e-01]],[[-1.41285166e-01,-7.87570655e-01,-4.68403876e-01],[2.65736073e-01,1.20445991e+00,6.73754275e-01],[-2.49738574e-01,-6.17512167e-01,-6.89494133e-01]],[[-6.57399297e-01,-7.56086767e-01,2.55674332e-01],[-3.74610782e-01,3.03453565e-01,-3.24170768e-01],[4.72077727e-01,2.03605592e-01,9.24975157e-01]],[[4.12888199e-01,-2.57054895e-01,9.48880091e-02],[-2.28638262e-01,4.32448566e-01,1.46083906e-01],[3.30877781e-01,2.08683237e-01,-4.56787318e-01]],[[-9.66013446e-02,-1.16601512e-01,-3.23435545e-01],[9.16084886e-01,-8.90289061e-03,6.11191809e-01],[4.39105511e-01,-1.46651417e-01,-2.37184837e-01]],[[-1.06485583e-01,3.64913456e-02,-2.78393626e-02],[2.65685797e-01,3.26594800e-01,-7.86264464e-02],[3.32362592e-01,6.06771529e-01,6.91408992e-01]],[[4.66866642e-01,1.52413592e-01,3.25077683e-01],[8.42853963e-01,1.89613700e-01,1.47799030e-01],[-2.99119532e-01,-8.47298563e-01,-3.56919050e-01]],[[-2.19556972e-01,-2.76033998e-01,1.10363826e-01],[1.42881917e-02,-4.51591522e-01,6.55128807e-03],[-5.77829778e-01,-2.49848679e-01,4.14352380e-02]]],[[[0.32530016,-0.30584317,-0.24411286],[0.1175686,-0.08875046,-0.39004013],[-0.06066065,0.34653285,-0.23204875]],[[-0.66572267,0.16291274,-0.19298334],[0.36934555,-0.29678297,0.11580221],[-0.00901898,0.11408991,-0.20371687]],[[0.20795996,-0.9066661,-0.7904463],[0.54238045,-0.25527784,-0.2695844],[0.56279784,0.5520405,0.00496885]],[[0.6441154,-0.381821,-0.6860296],[0.06889955,-0.0270679,0.6207713],[-0.34886986,-0.6214822,-0.60162437]],[[0.26040354,-0.3741268,0.03975657],[-0.82008797,0.14112633,0.58592206],[-0.7798593,0.41253757,0.06011292]],[[-0.4382399,-0.1873893,-0.57562095],[0.54157615,-0.13068372,0.33046415],[-0.92795384,-0.1354951,-0.45152187]],[[-0.11052299,0.3895071,1.2377757],[-0.30414826,-0.23952562,-0.30506134],[-0.6577335,0.12450089,0.6094207]],[[0.69941646,0.6258927,0.34137738],[0.4392877,-0.48702848,-0.04349888],[-0.09658453,0.89109224,-0.3444853]],[[-0.30874377,0.17385557,-0.14732951],[-0.213628,0.03754795,0.03977417],[0.9394923,-0.3149778,0.6100559]],[[0.19614206,0.27832416,-0.18977372],[0.4425128,0.38944516,0.03726812],[-0.08259309,0.5270078,0.26678392]],[[-0.6642252,0.56979823,0.39201277],[-0.5154666,0.13155693,-0.82416975],[-0.0417642,-0.02642204,-0.2404216]],[[-0.3127315,0.1835788,-1.4051001],[-0.2915461,0.12152772,0.00943852],[-0.14004788,-0.6420194,0.4039824]],[[-0.09213257,0.1306169,-0.04828547],[0.16476755,0.1260902,1.2656478],[0.04375799,-0.6461955,0.98279476]],[[-0.46489933,0.1316327,-0.48788553],[0.01996183,-0.9885449,0.454931],[-0.4840987,-0.838678,-0.07397677]],[[0.16348064,-0.12462796,0.33195588],[0.54711175,-0.00564356,0.06444006],[0.14644958,-0.49045277,0.51971215]],[[0.02604026,0.26917788,-0.03269245],[-0.57278204,-0.06249079,0.21204653],[-0.29128024,-0.08209638,-0.5580271]],[[0.20034643,-0.35693026,-0.06511208],[0.28759786,-0.38635027,-0.06209459],[0.08424295,0.21997216,-0.1569083]],[[-0.4458288,-0.37571913,-0.8677437],[0.22428182,0.16295429,-0.9238331],[0.3785974,-0.5826489,-1.2750896]],[[-0.01053262,0.1709053,-0.4648049],[-0.00820819,-0.28567553,-0.31819737],[0.02040756,-0.112257,0.56543165]],[[-0.23836236,0.15178059,0.26164317],[0.13899015,-0.09361495,-0.3824221],[-0.5124282,0.7333592,0.6882007]],[[0.50968456,-0.75428176,-1.1300931],[-0.117932,0.10518703,0.11523527],[-0.33873254,0.2764733,-0.20678826]],[[1.8200854,0.403085,0.20255373],[0.11468617,-0.39326155,-0.5678294],[0.25697508,0.21766643,0.30901843]],[[0.7518638,0.24269359,0.5892401],[-0.10984701,0.11572679,0.17261145],[-0.26480272,-0.21306905,0.16622676]],[[-0.80855364,0.50631326,0.13776465],[-0.86322486,-0.32506913,-0.60092586],[0.16828497,-0.5106098,0.3548328]],[[0.88413644,-0.50370175,0.57074],[0.16176507,0.51049584,-0.26351902],[-0.24019761,-0.03688625,0.41572833]],[[0.13137567,-0.5342509,0.56497645],[0.3894178,0.19350131,-0.76335764],[-0.04189489,-0.11161564,0.1048163]],[[0.289541,0.45688948,0.33123478],[-0.22651313,-0.04916335,0.31353757],[-0.5374202,-0.17834094,0.29137313]],[[0.53163517,0.07544401,0.1382633],[0.0618162,-0.17364174,-0.728841],[0.10218521,0.42494538,-0.23628193]],[[-0.36370045,0.06171325,-0.01148295],[0.11728223,0.49816272,0.41312015],[0.7154535,-0.4277215,0.29478392]],[[-0.06801469,-0.19457634,-0.30557626],[-0.2703279,0.08576225,-0.41547185],[-0.6826136,0.71143514,0.49662614]],[[-0.18981458,-0.07957865,0.4842001],[-0.0022465,0.13675345,0.7608009],[0.4161579,-0.24233179,0.45907742]],[[0.66291094,0.14983639,0.6979753],[0.51872694,0.97127557,-0.10667542],[-0.1452365,0.2888636,0.34851748]],[[0.17907669,-0.10684142,-0.26509053],[0.6365316,-0.16638151,-0.01166948],[-0.6166586,0.44779992,0.2675891]],[[-0.25268525,-0.08581199,0.24441469],[0.30424652,0.13252427,0.24573041],[0.24600326,-0.6266652,-1.3141347]],[[-0.44441196,0.1447127,-0.03929834],[-0.7363231,-0.56867397,0.20163675],[-1.145777,0.2991755,-1.2017553]],[[0.45996717,0.91600686,-0.53226167],[0.78430015,0.19398065,-0.69545805],[-0.2828254,-0.33730426,0.13626005]],[[0.03375721,0.5527733,-0.21194486],[-0.14275727,0.902586,0.32526338],[-0.242273,0.24941376,-0.16960719]],[[0.4232423,-0.20883444,0.6432816],[0.16448745,-0.67083275,0.32640567],[0.16526021,0.01341531,-0.12555109]],[[-1.4061807,0.35173914,0.71780807],[-0.309897,0.5982613,-0.10583046],[-0.19961002,0.37355477,0.28265658]],[[0.1005822,0.7039591,0.41513535],[-0.00239315,0.43181503,0.15104648],[0.41075867,0.3571324,-0.09608325]]],[[[-0.8655339,0.7901158,0.8535727],[-0.3331607,0.06441018,0.46632946],[-0.8240236,-0.265102,0.05521985]],[[0.07061584,-0.27029982,-0.10102373],[0.03894685,-0.7793261,-1.1273246],[0.68647254,-0.55256265,-0.12349025]],[[-0.421426,-0.25030214,-0.4942085],[0.53913444,-0.41870448,-0.40641612],[0.5440924,0.06041034,-0.68075216]],[[0.65224385,-0.37617952,0.00879379],[-0.09942511,0.4381961,0.61411357],[0.21854179,-0.47793323,-0.47226784]],[[-0.09772041,-0.07243991,-0.15679929],[0.13137348,-0.23790886,0.19799718],[0.39175022,0.32048893,-0.02002494]],[[0.06302303,-0.6877883,-0.4730653],[-0.11290112,0.4235643,-0.08514606],[-0.19769537,0.36906645,0.04584585]],[[-0.45710596,-0.42333463,0.36484355],[0.4776979,-0.09293363,0.43329975],[0.08315113,-0.04889106,-0.51727885]],[[-0.1617066,0.07706179,0.19731076],[-0.79276913,-0.27662554,0.79535174],[0.19423686,0.6285156,-0.24141026]],[[-0.56863284,0.05413403,0.26342753],[0.40337917,-0.5803304,-0.39704835],[0.0035923,-0.5070208,-0.5286868]],[[0.6655162,0.31327423,0.03622423],[0.3528487,-0.45918575,0.45370144],[-0.3546864,-0.24945146,-0.3091347]],[[-0.24252206,-0.4716874,0.61339855],[0.38718733,0.5193223,0.5678013],[0.05249905,-0.12917504,0.15025821]],[[-0.2911764,0.17623702,-0.60101146],[-0.3325103,0.13047944,-0.4109866],[0.5544343,0.428779,0.19915254]],[[-0.01700165,-0.09178872,-0.6887192],[-0.26805815,-0.6049808,-1.1198443],[-0.3855577,0.19392492,-0.15956117]],[[0.5467596,-0.26388866,0.33916292],[0.04887752,-0.65063107,0.88830376],[0.5473699,0.2929528,0.01068894]],[[-0.39282423,0.05305209,0.07845771],[-0.10766757,0.47998473,-0.918966],[0.14680277,-0.2433027,0.5361211]],[[-0.9812163,-0.03306992,0.35679644],[0.03057374,-0.08328559,0.06385066],[-0.9605649,-0.28873375,-0.46229324]],[[0.6736243,-0.369604,0.4133652],[-0.92917806,-0.34542283,0.42910454],[-0.3546429,-0.04962558,0.47074637]],[[-0.17975965,-0.5212597,0.10693315],[0.09368014,0.02670769,0.37626523],[-0.13318348,0.36549482,0.5222532]],[[0.6022096,0.35412467,-0.22472896],[0.3771872,-0.15378216,-0.7988354],[0.48342943,-0.275707,0.06913265]],[[-0.15893063,-0.6616826,-0.4374915],[0.3573288,-0.2069395,0.04702038],[-0.5668329,-0.6757224,0.49939224]],[[0.9517113,0.32578853,1.1621679],[0.07729863,-0.09759022,0.88501096],[0.21241887,0.2905797,0.3400471]],[[-1.9867711,-0.11956081,0.91124666],[-1.3010596,-0.04489966,-0.03037343],[-0.38148943,-0.08521377,-0.44299814]],[[0.3919163,0.14169773,-0.72675616],[0.24307482,-0.66948676,-0.5563949],[0.38561526,0.11546095,0.33426356]],[[0.16624963,0.5063947,-0.37033835],[0.02507375,0.6272489,-0.31540903],[1.0269512,-0.3621298,0.8733805]],[[-0.5442319,-0.49643698,0.18693455],[0.2775966,-0.5055608,-0.24423546],[-0.2256313,0.12566598,-0.52591425]],[[0.13972919,0.21948625,-0.38465178],[-0.15123583,0.7741515,0.5204599],[0.48084986,-1.0496106,0.6375785]],[[-0.77514416,-0.19719456,-0.28009865],[-0.12420155,-0.98536587,-0.62136406],[0.04505336,0.40746072,0.376637]],[[-0.89967984,-0.20299004,0.35584727],[-0.38036445,0.03482022,-0.07171971],[0.34710437,-0.19634716,0.3089219]],[[-0.14183319,0.28240043,-0.04314895],[0.05375062,0.1502804,0.5055006],[0.04331831,-0.28224623,-0.21383733]],[[0.68284243,0.0720119,0.4367262],[0.6229878,0.2184854,-0.10309039],[0.42050967,-0.0109818,-0.29343694]],[[0.13709287,0.15912393,0.61514306],[1.0545211,-0.38446924,0.25113207],[-0.48350623,0.15914235,0.8604491]],[[-0.17019248,-0.6513785,0.02635471],[-0.7773146,-0.4658754,0.14518791],[0.35368592,0.01856754,0.3260637]],[[0.49430448,0.17324407,0.6046137],[0.29491833,0.44931614,-0.45056403],[0.7730838,0.03689432,0.30565944]],[[0.06111547,0.12038188,0.5139283],[0.24976932,-0.01752859,0.15078636],[0.8752682,0.36426112,0.31116527]],[[-0.3126166,0.02129654,0.07795487],[0.1493531,0.5578029,-0.60426414],[-0.33561933,0.28073037,0.05781533]],[[-0.5730277,0.27317804,1.0474675],[-0.3910793,0.25523236,0.31252685],[0.11867177,-0.11957623,-0.3487494]],[[-0.3628555,0.07037383,-0.0652083],[-0.2434509,1.0517344,-0.2562966],[0.21417086,-0.08884048,0.7504755]],[[-0.03468985,-0.4602694,-1.0573779],[-0.14453772,0.37833878,0.08440573],[-0.29708573,0.4946487,0.48148265]],[[-0.31062385,0.15307666,0.07338537],[-0.12160581,0.29003006,0.17025693],[0.28340325,-0.39370134,0.07288241]],[[-0.29158434,-0.42316526,-0.4604117],[-0.5423861,0.46535605,0.16603383],[0.66207373,-0.15749444,0.35279614]]],[[[-1.25883818e-01,2.74055421e-01,4.47027564e-01],[-5.72675705e-01,-5.41040659e-01,-3.68079990e-01],[-3.13449115e-01,-1.86840892e-01,-1.73017196e-02]],[[-1.29468054e-01,1.80352911e-01,-3.63660336e-01],[7.73167834e-02,2.52517760e-02,-4.14782465e-01],[2.56636553e-02,8.30936670e-01,4.25903350e-02]],[[-2.11687367e-02,-1.37971062e-02,6.22780979e-01],[-2.30692357e-01,-3.72403592e-01,-1.44958436e-01],[-3.83885443e-01,5.30803025e-01,-3.87399405e-01]],[[-1.36931464e-01,-1.83475949e-02,-1.41918674e-01],[-3.45159143e-01,-4.28922251e-02,6.71244636e-02],[-5.70643783e-01,5.60196519e-01,-2.25292519e-01]],[[1.58579603e-01,2.62061924e-01,-6.90066814e-02],[-5.54704785e-01,-2.58168280e-01,-1.73935845e-01],[3.89419287e-01,-1.77313015e-01,-5.20769358e-01]],[[4.94118869e-01,1.25168979e-01,-5.50941408e-01],[-6.49135470e-01,5.16861260e-01,3.72701049e-01],[5.21405339e-01,-7.57713377e-01,1.31735131e-01]],[[-1.43141592e+00,1.28936279e-03,1.42192110e-01],[3.46903861e-01,-4.28640902e-01,-8.05405021e-01],[-1.24508333e+00,6.91075549e-02,1.00303791e-01]],[[5.97835541e-01,-2.41664127e-01,9.13811550e-02],[3.84489387e-01,-1.53208151e-01,1.09566140e+00],[-1.58293128e-01,-5.26097119e-01,-1.00757468e+00]],[[6.09431088e-01,-2.69122958e-01,-3.67870241e-01],[-5.03968000e-01,1.72795728e-01,-3.81342135e-02],[6.50771335e-02,8.01657617e-01,-6.12050235e-01]],[[3.40136945e-01,2.33917594e-01,4.68292445e-01],[3.17289799e-01,3.99839073e-01,-3.86411667e-01],[1.65445411e+00,2.04956949e-01,-8.18228841e-01]],[[-1.40811205e-01,6.35837257e-01,-2.98484385e-01],[9.93955195e-01,6.82113111e-01,-8.44868273e-03],[-7.00092409e-03,8.66946816e-01,-6.89887285e-01]],[[4.27218378e-01,-7.09955394e-01,-2.74621755e-01],[-4.43759933e-02,-2.76357144e-01,2.08876967e-01],[2.87658066e-01,1.84173733e-01,-6.24167621e-01]],[[-4.80234355e-01,4.71867204e-01,-3.21486056e-01],[-4.50510025e-01,-4.15285602e-02,-8.69814575e-01],[6.59971237e-01,2.30210841e-01,3.78238827e-01]],[[3.19907844e-01,-7.30568111e-01,3.82191777e-01],[2.78638899e-01,4.71314281e-01,8.79752815e-01],[-7.79504478e-01,-5.88024020e-01,-3.89716715e-01]],[[-1.72507226e-01,-4.75537807e-01,2.40969032e-01],[-6.87079847e-01,-1.67029709e-01,1.27160838e-02],[-2.20028505e-01,1.86139598e-01,6.72162354e-01]],[[-2.20815331e-01,-4.07095440e-02,2.40461543e-01],[1.07506178e-01,8.09602737e-01,-2.51147747e-01],[-7.73604393e-01,-7.55443335e-01,-5.58196783e-01]],[[5.15481234e-01,7.97108531e-01,-3.36402565e-01],[-4.03162003e-01,-3.29414904e-01,-1.31450713e-01],[7.28372633e-01,2.36699492e-01,-1.38468176e-01]],[[6.56836480e-02,-6.80308342e-02,9.54264924e-02],[-1.54133379e-01,1.94056123e-01,-6.00409135e-02],[-2.28034183e-01,2.25340754e-01,6.22146785e-01]],[[-4.40944582e-01,-6.39744429e-03,-3.98013860e-01],[-2.91791975e-01,5.36499977e-01,-1.60179317e-01],[-9.81664300e-01,6.55764267e-02,1.06763721e-01]],[[2.51887560e-01,2.21261218e-01,2.56473720e-01],[1.05495858e+00,6.07282296e-02,-4.49983813e-02],[-1.21223386e-02,9.73399207e-02,4.72549088e-02]],[[-2.48904184e-01,-5.41449726e-01,-8.69861022e-02],[1.49886101e-01,3.07488561e-01,-9.59014118e-01],[-2.26639286e-02,-4.91200864e-01,1.44239023e-01]],[[-6.14522398e-02,-4.33595389e-01,6.88782096e-01],[5.62821984e-01,-6.99330196e-02,3.87921035e-01],[7.14844942e-01,-5.26423156e-01,-2.47602969e-01]],[[-4.41276133e-02,-4.00216639e-01,3.32807191e-03],[3.57071191e-01,-3.15224051e-01,-2.96683550e-01],[4.08772171e-01,7.51014590e-01,-1.06412373e-01]],[[-3.41962516e-01,9.16487053e-02,-8.21292624e-02],[-5.37734866e-01,-4.79244679e-01,-3.30185205e-01],[-5.23104370e-02,-5.08070767e-01,-9.52648997e-01]],[[4.33939785e-01,-3.26460898e-01,1.16535991e-01],[-3.45468074e-01,-3.95566106e-01,-5.99053741e-01],[-4.28594947e-01,2.15204537e-01,3.13109517e-01]],[[5.89526176e-01,2.77271688e-01,-1.40236616e-01],[-3.97173762e-01,1.00724190e-01,-1.42486304e-01],[6.23819470e-01,-4.29778039e-01,-1.45927763e+00]],[[-3.28000247e-01,4.15759534e-01,2.63462067e-02],[-1.09443210e-01,-3.76791470e-02,-4.43238467e-01],[-2.57676780e-01,3.41087043e-01,-4.43519861e-01]],[[-6.21059716e-01,-6.14840925e-01,2.14294463e-01],[1.88278332e-01,3.24176848e-01,7.21082330e-01],[3.28119308e-01,3.91345769e-01,1.41815934e-02]],[[1.80526078e-01,-7.64249638e-03,6.28508478e-02],[-3.17804664e-02,-3.88924241e-01,-1.90034598e-01],[3.33602816e-01,-2.35862896e-01,-6.63088679e-01]],[[-4.04885292e-01,6.22325420e-01,-3.11547250e-01],[-7.22139955e-01,-5.27839065e-01,-8.66075575e-01],[4.15233582e-01,5.62546432e-01,4.95300628e-02]],[[-2.06709560e-02,9.18366611e-02,-1.88612312e-01],[-3.90636981e-01,-1.97653338e-01,-2.08935693e-01],[3.85897160e-02,1.31617367e-01,3.76766562e-01]],[[3.51331323e-01,-5.22147000e-01,-1.61780603e-02],[-4.02792394e-01,3.17361504e-02,-2.71249354e-01],[8.33784282e-01,9.76693444e-03,3.18539947e-01]],[[2.39526257e-01,5.16225994e-01,-4.00373548e-01],[9.72287953e-02,-2.36942858e-01,-2.51559138e-01],[-4.89384346e-02,4.25302386e-01,-1.60248220e-01]],[[1.37701482e-01,3.62397522e-01,3.78610879e-01],[9.35854763e-02,-5.31120121e-01,4.49783146e-01],[7.07819521e-01,2.00636312e-01,3.62876877e-02]],[[3.06943089e-01,1.40778378e-01,1.82522565e-01],[-6.58133030e-01,-5.46925604e-01,1.39280543e-01],[7.09074020e-01,-5.04217781e-02,7.10163653e-01]],[[3.31228250e-04,-3.34690362e-01,-3.48843873e-01],[2.87219673e-01,1.15947701e-01,-6.05434239e-01],[8.41727614e-01,1.16488069e-01,-6.71206653e-01]],[[-1.93618044e-01,2.86512017e-01,5.79522669e-01],[1.10111125e-01,5.63638806e-01,-7.66710281e-01],[2.59098619e-01,1.24130502e-01,2.84198374e-02]],[[5.51635802e-01,4.99692932e-02,3.46846133e-01],[-1.07160211e-01,1.80038348e-01,3.28083843e-01],[1.78176731e-01,-1.34602320e+00,2.67535210e-01]],[[4.46879536e-01,-2.95638800e-01,-7.87060261e-01],[5.43157816e-01,6.45810068e-01,2.35190362e-01],[-6.10340416e-01,-3.20737869e-01,-1.14290513e-01]],[[3.47483099e-01,-1.16176955e-01,-4.87195343e-01],[6.80905700e-01,-1.85930654e-02,3.01257581e-01],[5.92187420e-02,-2.40941886e-02,-5.37731290e-01]]],[[[-5.79376638e-01,6.79651618e-01,4.34123844e-01],[-1.52526215e-01,6.19711816e-01,-4.28384304e-01],[4.23259914e-01,1.15673348e-01,-1.66343786e-02]],[[-1.01597451e-01,-1.50579795e-01,3.92267793e-01],[-1.52517021e-01,4.58389223e-01,5.33864610e-02],[-1.05618812e-01,3.65398556e-01,2.18496963e-01]],[[2.87033051e-01,7.48381793e-01,1.74556509e-01],[-1.28508314e-01,-2.87876397e-01,9.51319709e-02],[-5.14471889e-01,1.87554613e-01,4.24659878e-01]],[[3.68538089e-02,-4.69337612e-01,8.68730187e-01],[-8.98046255e-01,-1.02203274e+00,-4.98858124e-01],[2.43071336e-02,-1.23550646e-01,7.00709760e-01]],[[-3.22428972e-01,1.15291715e-01,-3.91851753e-01],[1.68536276e-01,-6.37157440e-01,4.47274387e-01],[1.82144448e-01,2.28221983e-01,-7.46732831e-01]],[[3.00425380e-01,3.93907458e-01,1.57155424e-01],[-7.93783128e-01,-4.55815732e-01,-2.07633242e-01],[-3.91594589e-01,-7.67874420e-01,-1.87605903e-01]],[[-1.16613641e-01,-4.93656099e-01,5.12292325e-01],[9.41640958e-02,-5.13969243e-01,5.31722382e-02],[-5.52639484e-01,-1.68360129e-01,-4.47197437e-01]],[[-7.04842389e-01,3.51229161e-01,3.48632522e-02],[5.68799153e-02,3.05898249e-01,-8.56237292e-01],[-3.46553326e-01,-4.27104414e-01,-7.87283123e-01]],[[-1.00426710e+00,4.21609432e-01,-2.44648397e-01],[-3.37218314e-01,-2.91373640e-01,-1.12742402e-01],[3.08903068e-01,-3.14106286e-01,4.61629689e-01]],[[-9.40093994e-01,5.53609073e-01,-1.48244262e-01],[-1.12611413e-01,1.77179024e-01,3.17333460e-01],[-1.63228303e-01,-3.18976074e-01,3.36254090e-01]],[[-2.21353948e-01,-4.63728234e-02,-6.60824105e-02],[-3.97166491e-01,5.23713887e-01,-6.58374429e-02],[5.40805876e-01,-7.29389846e-01,-4.89430517e-01]],[[1.80956453e-01,9.25687790e-01,-2.50706226e-01],[5.86571731e-02,2.30090037e-01,9.22401845e-02],[-6.14940286e-01,2.01698076e-02,-4.01740789e-01]],[[1.04015090e-01,1.56251177e-01,-5.19930303e-01],[6.07400477e-01,-4.60283577e-01,-4.84504551e-01],[-1.50411949e-01,-4.46962982e-01,-7.56066561e-01]],[[2.55750835e-01,5.04723638e-02,-5.25509179e-01],[4.10729080e-01,-2.67435052e-02,1.59502640e-01],[4.48359609e-01,1.82622284e-01,-3.30339462e-01]],[[-5.83146214e-01,3.36592078e-01,-3.65630597e-01],[-8.13295841e-02,-6.88739195e-02,1.66801825e-01],[1.68215051e-01,-8.48970115e-01,-1.95423901e-01]],[[5.82559891e-02,1.21583603e-01,-4.35529113e-01],[-1.80453300e-01,-2.88208932e-01,1.26514673e-01],[-2.83092499e-01,4.94931817e-01,1.36090016e+00]],[[6.14061952e-01,-2.71207422e-01,-2.26019815e-01],[-1.41630903e-01,6.67766035e-01,3.22208673e-01],[9.85222042e-01,-3.40101689e-01,-4.79329497e-01]],[[4.03311402e-02,5.97895563e-01,-3.99340957e-01],[-1.11646922e-02,2.16418043e-01,2.33116165e-01],[5.10177672e-01,3.42232019e-01,3.51992518e-01]],[[-4.84230846e-01,7.38865972e-01,-9.59324986e-02],[-1.62992403e-01,-4.44584340e-02,4.45171416e-01],[3.86745065e-01,3.75146180e-01,5.42730510e-01]],[[2.45282471e-01,1.24905586e-01,3.63699466e-01],[4.25138772e-01,-6.05666459e-01,4.10781115e-01],[-3.31533670e-01,6.20188892e-01,3.21564704e-01]],[[5.77749670e-01,-9.35878903e-02,7.24335164e-02],[-2.66858965e-01,-7.09176242e-01,6.56333685e-01],[-2.02304378e-01,-3.18760306e-01,-6.31223898e-03]],[[-8.51833880e-01,-8.84669367e-03,-5.73831201e-01],[2.26746742e-02,-6.16083629e-02,-4.34680462e-01],[1.89908072e-01,9.48352814e-01,-5.05370617e-01]],[[-3.02902728e-01,-3.83423537e-01,8.63077879e-01],[-7.06456482e-01,1.28752440e-01,-1.88992187e-01],[1.65399760e-02,3.25671881e-02,1.75157592e-01]],[[5.29004745e-02,4.74927910e-02,-2.02518210e-01],[5.06451666e-01,-5.86895227e-01,-2.29698464e-01],[-3.00791562e-01,-2.22444847e-01,-1.97117642e-01]],[[-4.45256203e-01,1.69826075e-01,-1.82180807e-01],[-3.47595483e-01,-5.76150835e-01,-2.99925208e-01],[-1.54564545e-01,-2.20283866e-02,3.55052710e-01]],[[2.07835764e-01,-7.94852018e-01,-3.30978036e-01],[-8.66921782e-01,-6.36009336e-01,1.26336932e-01],[-5.00385582e-01,3.01845837e-04,1.43074423e-01]],[[-2.17288285e-01,-1.24251045e-01,6.41606331e-01],[-2.44880870e-01,-9.25815582e-01,-1.10615112e-01],[-5.22271812e-01,7.55152404e-02,7.20610797e-01]],[[-4.99378949e-01,8.22004005e-02,-3.30325782e-01],[3.02755356e-01,1.38447821e-01,2.15742394e-01],[2.80560367e-02,-1.81748778e-01,1.12273373e-01]],[[4.95784402e-01,-7.30876803e-01,-8.27676237e-01],[5.19315973e-02,1.77715555e-01,-5.75365424e-01],[-4.65892553e-01,-3.21714848e-01,9.01933908e-02]],[[9.85638559e-01,6.79846704e-01,1.71878740e-01],[-4.95757788e-01,-5.74556649e-01,1.00783145e+00],[9.53400016e-01,7.40371764e-01,3.23396325e-01]],[[2.07581922e-01,-5.95426708e-02,2.39577174e-01],[-5.87844968e-01,5.35096228e-02,3.14815760e-01],[4.02339809e-02,3.79317850e-02,3.50290656e-01]],[[3.43173742e-01,2.00467825e-01,-2.04696462e-01],[8.48388851e-01,2.04517636e-02,-3.27685207e-01],[7.08186090e-01,-9.88609552e-01,2.93019325e-01]],[[-7.22817302e-01,3.38859439e-01,3.47653896e-01],[7.50858188e-02,-6.55019164e-01,3.48563135e-01],[6.39514998e-02,-5.39286852e-01,5.45053303e-01]],[[-6.79313302e-01,-3.10492247e-01,1.00915635e+00],[1.02978654e-01,4.33343381e-01,1.58827111e-01],[5.98352849e-01,2.99239941e-02,5.35576403e-01]],[[-2.87824631e-01,-2.54076533e-02,-3.80078375e-01],[9.76979434e-02,2.16270342e-01,3.30434181e-02],[-4.40818757e-01,1.55552279e-03,1.31526485e-01]],[[4.67196822e-01,-1.72455728e-01,-2.13812903e-01],[-4.53443825e-01,2.36341015e-01,7.70536125e-01],[4.79869515e-01,-5.36995269e-02,-4.90322232e-01]],[[5.43297589e-01,-1.60852447e-01,2.21339598e-01],[7.97370911e-01,2.66277224e-01,2.03446567e-01],[-4.40737963e-01,3.40284288e-01,6.14113152e-01]],[[-2.32861474e-01,2.69470155e-01,1.71518922e-01],[-5.89593589e-01,6.28991783e-01,-2.07666472e-01],[2.92757452e-01,-1.43491399e+00,8.01180720e-01]],[[-5.77230632e-01,-2.65079111e-01,6.96975350e-01],[6.71135783e-01,-1.42660424e-01,1.02019861e-01],[-1.40800819e-01,3.01801771e-01,-2.90256202e-01]],[[1.57612175e-01,-2.46818468e-01,5.52660584e-01],[2.86188722e-01,-1.00392878e-01,3.01582843e-01],[-1.33191621e+00,1.91844016e-01,-1.14126527e+00]]],[[[1.69765323e-01,-6.31187558e-01,4.69459027e-01],[1.34824872e-01,1.73716396e-01,-8.77939314e-02],[8.11038196e-01,1.41607806e-01,8.11274443e-03]],[[-4.98866625e-02,-1.48814023e-01,-4.08579186e-02],[6.90321848e-02,7.70959482e-02,2.00028345e-01],[6.37787819e-01,1.09376825e-01,-3.72232616e-01]],[[-5.74883103e-01,1.50992153e-02,-4.92796838e-01],[1.20972261e-01,4.49193537e-01,-7.91073367e-02],[-9.13875327e-02,-1.48126334e-01,-2.24827453e-01]],[[-1.05775900e-01,-1.49327159e-01,-7.06340313e-01],[-3.71474698e-02,-3.12548816e-01,-2.51039453e-02],[3.89844418e-01,3.74800980e-01,3.04249048e-01]],[[-6.21220648e-01,-3.74742538e-01,-2.69385934e-01],[-8.69621560e-02,4.54880595e-02,-6.03389382e-01],[-3.85776699e-01,-8.57371837e-02,-7.03185201e-01]],[[-5.52381873e-01,-1.64501384e-01,-2.06523836e-01],[-1.12254228e-02,2.34650046e-01,3.05046916e-01],[-1.54688105e-01,-6.36991978e-01,4.40249413e-01]],[[-3.67115200e-01,1.67892665e-01,-6.69211924e-01],[-9.74696755e-01,-3.73626024e-01,-4.44536686e-01],[1.60926640e-01,5.46443343e-01,-1.47265244e+00]],[[5.06114900e-01,4.94142622e-02,2.38929614e-01],[3.86747330e-01,3.45040888e-01,6.13581777e-01],[4.86986041e-01,4.35340926e-02,-9.47218359e-01]],[[6.20326459e-01,-1.60081536e-02,-3.78470242e-01],[-2.01676842e-02,2.07870007e-01,5.67543209e-01],[-2.44054943e-01,-2.31978744e-01,3.99253249e-01]],[[-3.86966258e-01,2.79852003e-01,1.21985704e-01],[-1.05457958e-02,-4.75146323e-01,-4.62199971e-02],[5.36261261e-01,-2.78517395e-01,-4.29049790e-01]],[[-9.34362318e-03,-7.39637554e-01,-1.20908581e-01],[3.71359795e-01,-3.32897335e-01,3.96782696e-01],[-1.10228944e+00,-3.25202256e-01,1.50589705e-01]],[[-7.93555260e-01,-1.15102291e+00,3.68258357e-01],[4.64803800e-02,2.67407596e-01,-2.26555243e-01],[-5.33569276e-01,1.03811212e-01,2.46282101e-01]],[[-1.92106294e-03,-4.24127094e-02,-1.53648257e-01],[-1.09672987e+00,1.60946339e-01,7.95707762e-01],[-2.63808608e-01,-1.44601971e-01,2.09938943e-01]],[[3.55824828e-01,-6.35833502e-01,-1.47522530e-02],[-2.50890851e-01,-2.01284811e-01,-6.61613762e-01],[-3.49778622e-01,3.62818599e-01,-9.37044919e-01]],[[7.27865323e-02,-4.88622367e-01,3.84639680e-01],[-2.89950132e-01,5.16766250e-01,-6.46859705e-01],[-3.42626840e-01,-1.03291500e+00,7.63716459e-01]],[[8.74031484e-02,5.61455190e-02,1.15281627e-01],[-5.16868174e-01,-6.67717338e-01,-5.77838540e-01],[-1.28552645e-01,1.77946746e-01,4.56375144e-02]],[[1.02356106e-01,-1.43787637e-01,1.24916052e-02],[2.68867612e-01,5.46413541e-01,-4.00289178e-01],[5.63984752e-01,2.54704475e-01,-6.89828336e-01]],[[-7.88435414e-02,-1.94925621e-01,1.80068702e-01],[1.65114701e-01,-4.47049499e-01,-4.51103486e-02],[1.87958598e-01,-1.09336650e+00,-8.09044003e-01]],[[1.48606271e-01,1.37488050e-02,-7.09130690e-02],[-1.53575331e-01,-2.00286150e-01,3.16551089e-01],[5.78672707e-01,6.52722538e-01,1.93728924e-01]],[[-1.36899248e-01,-5.53149104e-01,4.07704562e-01],[-4.89388078e-01,-4.09551978e-01,5.72782099e-01],[6.18753254e-01,5.84088862e-02,1.02732408e+00]],[[-6.32271528e-01,-6.53327525e-01,-9.47267294e-01],[-1.71013311e-01,3.94208282e-02,-3.30860198e-01],[-3.09324354e-01,-7.89604429e-03,-6.64119244e-01]],[[2.34751374e-01,6.51969731e-01,8.05903435e-01],[1.62685104e-02,-3.02454531e-01,3.09887439e-01],[-9.22435671e-02,-1.94466531e-01,-2.92360127e-01]],[[3.08552474e-01,-7.88680464e-02,8.88471246e-01],[-4.69479471e-01,2.78991044e-01,8.00740123e-01],[-9.87936333e-02,-1.15635626e-01,4.56666023e-01]],[[5.93498290e-01,-5.30643106e-01,-5.08439727e-02],[-2.31554121e-01,1.13474704e-01,2.08707705e-01],[-1.04828469e-01,4.69126478e-02,-4.32276636e-01]],[[9.66458440e-01,3.14742178e-01,9.15497363e-01],[2.56220907e-01,-2.85870522e-01,8.46608460e-01],[7.12267816e-01,1.14190027e-01,3.99646550e-01]],[[-8.44893381e-02,1.22373641e-01,-1.49881691e-01],[1.36103302e-01,1.95802391e-01,-6.09540761e-01],[-6.56170070e-01,6.08572125e-01,5.84926546e-01]],[[1.68652132e-01,1.73002690e-01,8.34170699e-01],[1.51548252e-01,-5.50575927e-02,5.23555815e-01],[-7.93506578e-02,9.88472641e-01,4.22226369e-01]],[[3.92419219e-01,9.78186876e-02,-1.35386422e-01],[-5.48775755e-02,7.77371049e-01,-1.70304328e-01],[5.55498362e-01,7.57115543e-01,1.03495789e+00]],[[4.02852893e-01,-3.06329161e-01,-1.45323908e+00],[-7.31177256e-02,4.94707078e-01,-3.94179910e-01],[3.93285692e-01,-6.92969918e-01,1.80193692e-01]],[[-4.95368302e-01,-7.89124370e-02,-3.62646073e-01],[-8.59512210e-01,-6.92144558e-02,-9.05252695e-01],[-1.27789152e+00,-1.81275979e-01,9.51710939e-02]],[[8.02372754e-01,4.52822477e-01,-9.42525506e-01],[4.69862074e-01,2.08075657e-01,6.83288693e-01],[3.28997336e-02,-1.45504419e-02,-6.14328325e-01]],[[-4.50452834e-01,-3.45134914e-01,3.57775122e-01],[1.36171669e-01,-3.06238145e-01,-7.32819796e-01],[7.44202435e-01,4.08299536e-01,-1.69225141e-01]],[[9.85296890e-02,1.82196423e-01,-8.99467647e-01],[3.33432317e-01,2.21738383e-01,-6.91700578e-01],[-2.14468822e-01,3.28828126e-01,1.07264869e-01]],[[1.36573064e+00,-5.12561575e-03,5.64434707e-01],[6.74025863e-02,7.55926490e-01,-2.81940460e-01],[-2.40522534e-01,6.01938486e-01,-1.02150925e-01]],[[-2.76449472e-01,-1.17082477e-01,1.74537569e-01],[-8.63400400e-01,-2.63060600e-01,-2.34620616e-01],[1.57475471e-01,6.79402500e-02,-8.97740185e-01]],[[3.40587318e-01,-3.81060839e-01,-2.72614002e-01],[-4.60547596e-01,-1.23311281e-01,2.54978627e-01],[4.40966666e-01,-3.85206565e-02,-1.48216978e-01]],[[2.68670231e-01,-3.60375643e-01,3.98333825e-04],[-2.82290578e-02,-1.67178094e-01,-2.10560083e-01],[-3.27899009e-01,-1.59886211e-01,-3.29866081e-01]],[[6.59654558e-01,-2.79132724e-01,2.97879755e-01],[-2.96737719e-02,-2.55942225e-01,6.57006204e-02],[6.21373773e-01,-3.34962815e-01,9.73440632e-02]],[[-2.86467165e-01,-3.97131294e-01,2.77028456e-02],[4.16024625e-01,-3.38305980e-01,6.24224722e-01],[-1.65234491e-01,-5.78155994e-01,6.19827271e-01]],[[-1.40906915e-01,-9.64245275e-02,-3.25236097e-02],[5.27645171e-01,3.96144509e-01,3.26023370e-01],[-1.13994151e-01,-5.51426038e-02,-6.59795880e-01]]],[[[-0.01346061,-0.5904002,-0.35331386],[-0.08994044,0.21879657,-0.54559386],[0.24790199,-0.7897986,-0.49905872]],[[-0.00452727,-0.24758472,0.34857544],[0.24047953,0.00889703,-0.5010653],[0.81886804,-0.46707657,-0.07377157]],[[0.09876372,-0.44207308,0.00191259],[-0.3325042,0.05855924,0.09668143],[-0.02261178,0.02350309,0.28737402]],[[-0.41138878,0.8911743,0.2568522],[-0.23224594,-0.1871823,-0.34369624],[-0.19000599,0.2945058,-0.12469542]],[[0.69507307,0.07788329,-0.34050858],[-0.4321335,0.23258361,-0.8567609],[0.04118523,0.16010717,0.2586116]],[[-0.39669847,0.6451624,-0.73998564],[0.10123427,-0.42303458,-0.3692145],[-0.14917952,-0.3619431,-0.63893217]],[[-0.14135164,-0.20112765,-0.19346347],[-0.6244574,0.01431038,-0.5292564],[0.8445877,-0.82089186,-0.44778764]],[[-0.38190567,-0.8821218,0.10467786],[-0.74391514,0.19250934,-0.35195634],[0.5399266,0.3264583,-0.19273308]],[[0.37904152,-0.4417509,0.03127738],[0.08423531,0.326048,-0.23694411],[1.0607817,-0.36201054,0.18486246]],[[0.22069001,-0.23577932,0.41153654],[-0.11163303,-0.46323404,0.11185475],[-0.24626105,0.27596804,0.06396767]],[[0.4238627,0.0105359,-0.02094848],[0.2300281,-0.26572362,-0.28910646],[0.6481724,-0.5269941,-0.30318382]],[[-0.11082372,0.11378538,-1.1404715],[-0.63869673,-0.35700247,-0.60875195],[0.05968662,0.4170897,-0.52075523]],[[0.09291871,-0.3385456,0.3645202],[0.09892759,0.34283772,0.8384584],[0.6034084,-0.23201363,-0.11902848]],[[0.3374241,0.4240928,0.18832797],[-0.50734764,-0.09753124,-0.29334053],[-0.05710608,0.48900282,-0.30538428]],[[0.16922534,-0.48962128,-0.17205526],[0.38895085,0.45976162,0.24330871],[-0.18034309,-0.69474024,0.21411563]],[[0.03952139,-0.07958119,1.1673824],[-0.35296008,0.85137904,-0.0911687],[0.6720807,-1.0276608,-0.2236246]],[[-0.9399178,0.04504642,-0.12346292],[-0.22596613,-0.86746526,-0.43269414],[0.21836524,-0.55150986,-0.37373474]],[[0.28585005,0.06495439,0.450975],[-0.12046756,-0.51961476,-0.4886027],[-0.26940665,0.15698594,-0.4067838]],[[0.20667669,-0.01293998,-0.40396115],[0.0810798,0.23773427,-0.47824076],[-0.99472284,-0.20528287,-0.15869977]],[[-0.22837158,0.204549,0.5271625],[-0.08346854,-0.16440994,1.2798946],[-0.3946047,-0.3840636,0.9112811]],[[-0.01538676,-0.1545209,-0.3393501],[-0.4573343,-0.18598153,0.17059462],[-0.6602588,0.32651302,0.02691906]],[[-0.8039369,-0.5438941,-0.07992733],[-0.35113737,0.02297326,0.12042045],[-0.93074155,-0.08221765,0.38079694]],[[-0.10754199,-0.5046905,0.3593203],[0.36386257,-0.7355759,0.39291885],[-0.9060995,0.18286075,0.23219629]],[[-0.75416255,0.03615757,0.2930373],[-1.2417018,-0.191389,0.99888754],[0.04035829,-0.58774084,-0.30477348]],[[-0.20648877,-0.46772608,0.4851387],[0.06241594,-0.05839524,0.9769237],[-1.2934797,0.6381366,-0.35737336]],[[-0.3053697,-0.57615286,0.8178647],[-0.05134559,-0.15205987,-0.7977009],[-0.49536148,-0.39417058,0.11741834]],[[0.10933825,0.05204797,0.2734579],[-0.07726414,-0.6990663,0.8437683],[0.25496444,0.00452727,0.357138]],[[-0.00820915,-0.25567532,0.6180222],[-0.6642776,0.12663883,-0.01460581],[0.38889602,-0.05388933,0.16266923]],[[-0.5135932,0.5693333,-0.80861425],[-0.24330708,-0.21483839,0.28393888],[0.07363368,-0.32988015,0.13801563]],[[-0.08556741,-0.776569,-0.03133726],[0.0182365,0.61204666,-0.17147198],[0.29015782,0.02223435,0.22230214]],[[-0.42264876,0.23112158,0.28546405],[-0.02941763,-0.06552376,0.20192419],[-0.41094992,0.31941503,0.00505068]],[[-0.04801543,-0.4665929,0.06188342],[-0.86820644,0.4293981,0.21558467],[-0.7837484,0.13258344,-1.3102248]],[[-0.2549597,-0.21892463,0.42579812],[-0.38965893,-0.11948469,0.33300096],[-0.12302392,-0.38447347,0.26775673]],[[-0.31419304,-0.30546898,-0.18857846],[0.07590936,-0.17447616,0.64180416],[0.25274768,-0.86093056,0.15629256]],[[0.28898638,0.25174206,0.5749121],[-0.01140528,0.30952907,-0.01535378],[-0.6026942,-0.09133946,-0.10259177]],[[-0.5931298,-0.00690335,-0.03230482],[0.8202877,0.76842153,0.3509267],[-0.72496504,-0.04951817,-0.573213]],[[0.33345369,0.500234,-0.15883563],[0.51703644,-0.45416343,0.5096805],[-0.05228269,-0.77374756,-0.9669235]],[[0.443284,-0.10931976,1.0879316],[0.07156088,0.23982178,0.01378448],[-0.76798254,0.7101808,0.05515359]],[[-0.28851038,-0.449109,0.12091169],[-0.586882,0.52339053,0.49113575],[-0.1451288,-0.17978941,0.04077823]],[[-0.70066065,-0.50927943,0.79464597],[-0.4770533,0.45680645,-0.32599914],[0.880229,-0.21801002,0.3048359]]],[[[0.8887647,0.45876002,0.4397471],[0.6288415,-0.1736643,0.08392289],[-0.19796835,-0.22100878,-0.5282879]],[[-0.35519773,0.12459187,-0.21416795],[-0.3777724,0.2613648,-0.13688137],[-0.14389873,0.12811154,0.0342399]],[[0.2627473,0.39454842,0.34238765],[-0.19868132,-0.02555694,0.17963168],[-0.1616539,-0.0991848,-0.30268082]],[[0.19311047,0.6788839,0.3519891],[-0.3239509,0.48979637,0.15745081],[0.2830542,0.22712429,-0.2408408]],[[0.4279492,-0.12186663,0.2959118],[0.624999,-0.16384202,0.9049828],[-0.07968868,-0.03656234,1.0331653]],[[0.41135043,0.38915,-0.22843105],[-0.04389411,-0.40708524,0.44530836],[0.08442183,0.43735653,-0.5940983]],[[-0.03033609,-0.29946077,-0.09466492],[0.24557973,0.15718652,0.6811367],[-0.17476553,0.19264533,0.8768431]],[[0.24528569,0.19680588,-0.02215126],[-0.3168108,0.11378749,-0.07279062],[0.05592142,-0.40676236,0.40169764]],[[-0.1159665,-0.04802081,-0.42065117],[0.52378756,-0.01803473,-0.21985896],[-0.45464262,-0.26035765,-0.23306175]],[[-0.00456853,0.67481816,0.3185223],[-0.21695988,0.49511805,0.27817586],[-0.691463,0.41484818,-0.3308533]],[[-0.1588271,0.01040808,0.24529044],[0.29565352,-0.25870055,0.1557246],[0.01576687,-0.2097021,-0.28441298]],[[0.4117813,0.6384403,-0.09892696],[-0.12082906,-0.2736142,-0.04268561],[0.7055223,-0.61128694,0.152882]],[[0.15773785,0.18166071,0.2654568],[0.16201676,0.10975489,-0.17942615],[0.3534945,-0.05349117,-0.31150767]],[[0.08972768,0.23723821,0.06742766],[1.027588,0.36060098,0.4276741],[1.2112095,0.6655912,0.22584039]],[[0.16550964,-0.3570869,-0.05892462],[0.55443287,-0.3295463,-0.32685515],[0.07432346,0.00312658,0.04836088]],[[-0.09866999,0.31377593,0.48707357],[0.637379,0.364909,0.5233123],[0.47815108,0.45668086,0.68043256]],[[-0.68647575,-0.07180636,0.1551728],[-0.02488308,-0.0370133,0.32147086],[-1.6150781,0.32479835,0.4078623]],[[0.2117124,0.08846173,0.05278793],[0.29879785,0.60418344,0.3662123],[0.10628397,0.47911987,-0.36171302]],[[-0.2959915,-0.11605355,-0.16005093],[-0.44133064,-0.1719918,-0.23838548],[-0.1066309,0.32835776,-1.0688808]],[[-0.21730942,-0.06350769,0.47179088],[0.08509257,0.6629214,-0.5147248],[-0.38726613,-0.49297276,-1.0519654]],[[0.80442107,0.54121256,1.8315828],[0.37316987,0.15551117,0.08683178],[0.54359746,0.12743978,-0.09320384]],[[-1.1120766,-0.0262689,-0.40881702],[-1.0131125,-0.02961631,0.08178628],[0.08404743,0.04423471,-0.11545958]],[[-0.6669089,-0.1730962,-2.0276074],[0.10598758,-0.20225023,-0.44332978],[0.56228524,0.21085244,-0.23917551]],[[0.47924578,-0.3326075,-0.36714557],[0.5910938,0.40518326,-0.22486475],[0.5004655,0.21253076,0.21719977]],[[-1.1900403,-0.29827943,-0.48481354],[-0.5007262,-0.15397973,0.01316257],[-0.35363245,-0.09202548,-1.0029842]],[[-0.08690363,-0.5013724,-0.16811727],[0.09480851,-0.2778096,0.38707644],[-0.13912569,-0.70136076,0.2711189]],[[0.071219,0.2768998,0.30028746],[-0.01986156,0.31865764,-0.25577042],[0.08818445,0.27279636,-1.0047293]],[[-0.09274139,0.25532648,0.2640146],[0.10836618,0.02514445,-0.66821826],[-0.4603022,0.1763762,-0.37109032]],[[0.62284654,0.9992533,1.1038101],[-0.24767822,-0.04702059,0.4885569],[-0.40788695,-0.09737187,0.3605199]],[[0.46850815,-0.24963963,0.09907228],[0.10648991,-0.16576876,0.3335695],[0.5222955,0.28389558,0.938524]],[[0.06842579,0.30735227,-0.23166814],[0.14673744,-0.22367497,0.15172067],[0.46786553,0.5438611,0.17730181]],[[0.01140851,-0.1029368,0.17260067],[-0.7129218,0.6440827,0.26283953],[-0.10063878,0.7352718,0.07871339]],[[-0.5963379,0.3052951,0.3042887],[-0.25027826,0.05516065,0.11828764],[-0.2334141,0.36015555,-0.45909414]],[[-0.0038292,0.17895319,0.7104984],[-0.9978228,-0.5950136,0.3153368],[-0.82628864,0.05529055,1.552587]],[[0.8561001,-0.419593,1.0205526],[0.0432961,-0.19889016,0.9214423],[0.43737808,-0.3493076,1.0399696]],[[-0.20181634,-0.08663728,0.1817943],[-0.4700233,0.1432118,-0.00896519],[-0.06126066,-0.23997453,-0.77659684]],[[0.25806162,0.2644117,-0.05288065],[1.148687,-0.14066352,-0.0666625],[0.09145259,0.01272782,-0.11969949]],[[-0.2632029,0.21822676,0.48300052],[0.3021147,0.7035471,0.08656539],[-0.2840412,-0.34827253,0.6160004]],[[1.3837527,0.23891494,0.12961346],[1.1949006,-0.2828844,0.20363276],[1.1929221,-0.4640609,0.58969325]],[[-0.51591676,-0.19769128,-0.46891138],[-0.0094681,-0.01612886,-0.03842701],[0.24199015,0.48994604,-0.03030648]]],[[[-6.08830571e-01,-4.15891916e-01,-2.95255631e-01],[3.02029759e-01,8.91318619e-02,1.50709987e-01],[-6.10245287e-01,1.01636864e-01,6.29549563e-01]],[[-3.82270604e-01,-1.53154999e-01,-1.08590811e-01],[-6.09855473e-01,3.23112935e-01,4.75501478e-01],[-2.93905325e-02,3.96430463e-01,-8.23140204e-01]],[[1.04371130e-01,-4.02479053e-01,-4.58512336e-01],[2.65998572e-01,1.05496883e-01,2.66065937e-04],[4.25458886e-02,-5.23927152e-01,-5.36686838e-01]],[[-3.40271682e-01,6.01555884e-01,3.93225104e-01],[-4.20501798e-01,3.03811610e-01,-3.05694848e-01],[-3.03122669e-01,3.76359552e-01,3.56499016e-01]],[[3.63062769e-01,-5.36510944e-01,6.41550422e-01],[4.80215102e-01,-5.22972763e-01,1.31848499e-01],[3.70805502e-01,3.01460385e-01,-1.69226006e-01]],[[-1.30192876e-01,-1.80302992e-01,-9.95344102e-01],[5.51717281e-01,-4.86789674e-01,-9.87195313e-01],[2.94255912e-02,-2.29848966e-01,3.79311144e-01]],[[-3.40576023e-01,-1.43416330e-01,5.00391424e-01],[-6.44333303e-01,6.16161942e-01,-1.66687325e-01],[2.45376274e-01,-8.38891506e-01,-6.93509221e-01]],[[1.60910308e-01,1.58758968e-01,2.87862301e-01],[5.37180118e-02,2.71300405e-01,-8.50145593e-02],[8.71018529e-01,-4.64469224e-01,2.01913923e-01]],[[-3.35284412e-01,-9.06659365e-01,-2.58622551e-03],[-8.99091125e-01,1.81210995e-01,-4.63809460e-01],[7.38296285e-02,3.00812364e-01,-3.11275482e-01]],[[4.03617769e-01,7.80918777e-01,-4.10770476e-01],[3.63227613e-02,-2.24325974e-02,1.91556334e-01],[-1.06189871e+00,-3.55689973e-01,-3.96561325e-01]],[[1.96626827e-01,-1.29206464e-01,4.11824942e-01],[-1.46980196e-01,-5.84220529e-01,6.57792628e-01],[1.32382244e-01,-4.14901257e-01,3.81584056e-02]],[[2.51751244e-01,2.95677811e-01,-3.95647347e-01],[-3.41641217e-01,4.89153452e-02,-4.82954800e-01],[2.55012423e-01,9.42634881e-01,5.08061936e-03]],[[-1.40784815e-01,8.08850825e-02,-7.72016287e-01],[-3.35648954e-01,3.68474841e-01,6.79698765e-01],[1.25041649e-01,-2.41368152e-02,5.38312137e-01]],[[1.05073166e+00,5.90907149e-02,1.98942766e-01],[3.16097736e-01,-7.88980275e-02,-3.66531104e-01],[7.01199532e-01,7.27290064e-02,4.63150665e-02]],[[4.06162411e-01,4.05606151e-01,7.21748292e-01],[-3.51589650e-01,3.32161635e-01,-2.81714499e-01],[-4.83403683e-01,-7.09064603e-01,-6.41156077e-01]],[[-5.88747598e-02,7.26117268e-02,-1.16317123e-02],[-3.15869540e-01,-1.46761286e+00,-1.22044757e-01],[6.25345111e-01,8.21174681e-01,6.32455051e-01]],[[-2.50473440e-01,3.06536824e-01,-3.41333270e-01],[-1.31177017e-02,3.27143967e-01,-4.02237177e-01],[-6.13339365e-01,-2.08700210e-01,-7.70555794e-01]],[[1.71122432e-01,-7.72523403e-01,4.72180992e-01],[1.48866907e-01,-7.46691227e-03,4.48370159e-01],[4.15663660e-01,3.32606941e-01,8.91554296e-01]],[[2.00922564e-01,4.17682022e-01,-3.00189674e-01],[3.26785564e-01,-6.38344884e-02,-2.33710021e-01],[5.05339801e-01,-3.91118787e-02,1.73655689e-01]],[[-1.03201580e+00,8.53610504e-03,-5.00965156e-02],[-3.21288645e-01,3.58039469e-01,3.82673651e-01],[-2.27383554e-01,1.54328123e-01,-9.87482905e-01]],[[5.73000073e-01,8.67127359e-01,2.39671320e-01],[4.65955973e-01,9.34803113e-02,1.87197044e-01],[2.86257833e-01,-2.16788724e-01,-4.72185433e-01]],[[-1.17704809e+00,-4.47842479e-01,1.68822229e-01],[-2.52155125e-01,-2.21590355e-01,-4.01914679e-02],[-1.37569502e-01,-3.77118468e-01,3.54361027e-01]],[[1.00561261e-01,6.04067557e-02,-5.55590332e-01],[3.63602221e-01,1.90755859e-01,-5.97556055e-01],[5.00980556e-01,3.57626200e-01,4.25029127e-03]],[[2.73708254e-01,4.36589532e-02,-2.61381656e-01],[6.76099896e-01,3.89462590e-01,3.28734279e-01],[8.66665661e-01,-3.77420783e-01,7.73707569e-01]],[[-4.68065917e-01,1.54557124e-01,1.73655584e-01],[-2.95777857e-01,2.25509360e-01,3.09351981e-02],[-9.48089540e-01,7.42764329e-04,-5.89303315e-01]],[[3.79834026e-01,-2.68886715e-01,7.71715939e-02],[2.11910427e-01,-1.35606110e-01,2.37972155e-01],[5.13098300e-01,-1.86458081e-01,-2.62820274e-01]],[[7.08049089e-02,-2.16774374e-01,-6.99463427e-01],[-2.40663007e-01,1.20437354e-01,-7.22410679e-02],[-1.20174987e-02,-2.30099522e-02,7.43809760e-01]],[[5.40937185e-02,8.59209955e-01,3.96639258e-01],[1.55516431e-01,-5.56951463e-01,3.21890414e-01],[1.45126253e-01,5.37212014e-01,-7.53368258e-01]],[[-1.06395409e-02,2.04494491e-01,4.12037343e-01],[9.77913514e-02,4.21179049e-02,1.90925434e-01],[-2.58696135e-02,5.37523702e-02,7.24075854e-01]],[[6.00482583e-01,4.03277904e-01,1.15689680e-01],[-1.15951002e-01,4.16474044e-01,4.64843474e-02],[4.48110431e-01,-2.03746840e-01,2.09037602e-01]],[[-3.51113647e-01,-1.70173079e-01,3.13528657e-01],[2.27972463e-01,6.01982236e-01,3.28653693e-01],[8.47895801e-01,-1.53896436e-02,1.66538619e-02]],[[-9.52628255e-01,-1.00578952e+00,2.62194306e-01],[-1.20556213e-01,-1.02975500e+00,-3.45644653e-02],[3.95814568e-01,4.41457570e-01,-3.58782947e-01]],[[1.80605680e-01,8.50202888e-02,9.97739613e-01],[-1.82068571e-01,5.86159647e-01,-2.54654527e-01],[-6.33427620e-01,3.66320670e-01,-5.86527765e-01]],[[-2.78512776e-01,6.54370198e-03,-3.11848968e-01],[7.35778749e-01,7.77701735e-02,3.32974821e-01],[-3.42386097e-01,8.29214081e-02,7.19816089e-01]],[[-5.28533936e-01,1.54096454e-01,2.22825855e-01],[1.08921862e+00,-4.48554665e-01,9.62177157e-01],[4.43023583e-03,3.03755790e-01,-1.98693335e-01]],[[-7.01042056e-01,-2.89971344e-02,3.74295861e-01],[-2.96502709e-01,-8.67442548e-01,8.13550130e-02],[-7.16433346e-01,-5.85759938e-01,-1.18226671e-04]],[[2.14922473e-01,3.62044692e-01,1.64582878e-01],[-2.23509759e-01,6.01449668e-01,-4.62448090e-01],[3.54563594e-02,7.92136014e-01,-6.20752096e-01]],[[-1.08687527e-01,2.00451046e-01,2.77226478e-01],[-3.46910864e-01,5.19237332e-02,3.90096724e-01],[-1.26966691e+00,-4.08656985e-01,-5.58854878e-01]],[[-1.17374994e-01,-3.78200024e-01,-5.13208508e-01],[1.78946450e-01,-2.47531772e-01,3.12628835e-01],[-3.05546224e-01,-4.84558433e-01,-9.51711833e-01]],[[1.86340034e-01,-4.84167874e-01,-2.20898151e-01],[1.81775615e-01,5.84559977e-01,-1.72288734e-02],[-4.84592766e-01,-4.38904315e-01,-2.71356791e-01]]],[[[7.59371042e-01,3.03270221e-01,-3.99235308e-01],[-7.30956137e-01,-2.23048944e-02,-5.94989538e-01],[3.66523340e-02,-3.42723280e-01,-5.50696075e-01]],[[-1.30865991e-01,1.69392616e-01,-1.09414153e-01],[-2.05385953e-01,-3.43282282e-01,-3.82946014e-01],[3.14802468e-01,-1.68808565e-01,2.92953968e-01]],[[6.90281212e-01,-9.26010236e-02,-3.09924930e-01],[-4.41570163e-01,-8.87159526e-01,-2.05839261e-01],[8.96782950e-02,-1.01737952e+00,2.39969894e-01]],[[5.62994659e-01,1.96139961e-01,-3.80404294e-01],[3.98880750e-01,5.96734583e-01,2.72282094e-01],[6.06659055e-01,6.59873605e-01,-5.99669337e-01]],[[1.54305160e-01,1.05567262e-01,-5.94714358e-02],[4.66241837e-02,1.59892842e-01,2.64272392e-01],[2.22429648e-01,2.90201634e-01,-3.75989139e-01]],[[7.65381232e-02,-2.23852590e-01,-5.55935264e-01],[2.77595609e-01,-3.62850249e-01,1.71447292e-01],[-1.19561203e-01,-3.55253741e-02,5.38616955e-01]],[[-2.54242331e-01,9.38465297e-02,-5.61206996e-01],[-5.41775346e-01,-4.20998372e-02,4.33907926e-01],[8.40921521e-01,-3.11090052e-01,1.61418185e-01]],[[7.45650083e-02,-2.80324876e-01,-7.95671940e-02],[-2.29115710e-01,5.68853877e-02,-8.20488691e-01],[2.32965022e-01,-2.46977031e-01,-1.11916400e-01]],[[1.39472326e-02,-1.04888119e-02,2.54392415e-01],[-2.88729012e-01,-5.71977556e-01,7.50614047e-01],[5.60190678e-01,9.48691219e-02,2.81763583e-01]],[[-5.54293096e-01,-6.07833743e-01,-7.80902743e-01],[-3.73305768e-01,-2.76695698e-01,-6.99734688e-01],[-8.71526420e-01,-1.45161533e+00,-4.14185017e-01]],[[6.60732865e-01,-1.87774435e-01,-4.10701632e-01],[4.45011586e-01,4.56738621e-01,2.02518538e-01],[-4.65079635e-01,1.35790762e-02,-2.07213014e-01]],[[4.88014817e-01,-8.73113573e-01,1.50990725e-01],[7.31176317e-01,-9.63507220e-02,-1.11493051e-01],[-3.16142529e-01,-1.48057073e-01,7.07516000e-02]],[[9.02470425e-02,4.92288843e-02,-6.32173538e-01],[-1.22250125e-01,3.45339060e-01,-6.82008803e-01],[-7.18044877e-01,3.72630686e-01,7.87164178e-03]],[[-4.30930883e-01,1.78505957e-01,-6.52991056e-01],[-2.86703408e-01,-3.10535908e-01,2.20354438e-01],[5.08775227e-02,-9.38729882e-01,-6.14502072e-01]],[[9.48385298e-01,-5.20242274e-01,-6.36559665e-01],[-2.22687498e-01,1.71887636e-01,1.09396249e-01],[1.11395679e-01,1.69440866e-01,5.26671410e-01]],[[-6.19997233e-02,-1.95725381e-01,3.01574357e-02],[3.78016800e-01,4.37815607e-01,1.08510172e+00],[2.32453600e-01,6.99204803e-02,1.87671959e-01]],[[8.05174708e-02,-2.83176661e-01,2.61959374e-01],[-5.07277012e-01,-6.19488418e-01,-2.41645992e-01],[-1.82903349e-01,-2.04237953e-01,3.56119446e-04]],[[1.54099822e-01,-2.58952349e-01,-1.56193584e-01],[-2.91615486e-01,1.57627612e-01,-2.20954910e-01],[2.90371448e-01,-7.84874782e-02,-3.62910986e-01]],[[4.02168512e-01,-3.94620091e-01,-9.82250422e-02],[2.71102458e-01,-3.68107170e-01,-7.65465617e-01],[4.67607021e-01,5.07056892e-01,-1.15630996e+00]],[[-1.42558858e-01,1.61297217e-01,1.05357476e-01],[7.90311396e-02,2.26347856e-02,1.27427801e-01],[2.52818167e-01,6.19886935e-01,8.20453405e-01]],[[-3.52884948e-01,1.03292607e-01,-2.57184029e-01],[-2.55415827e-01,1.40994892e-01,-9.26533163e-01],[-9.22322199e-02,-1.50971217e-02,-5.15123487e-01]],[[8.25068891e-01,-1.92689031e-01,2.95368195e-01],[-4.74829823e-02,-6.17592642e-03,-2.61078268e-01],[-1.39235437e-01,1.28054380e-01,1.20103252e+00]],[[-4.37847406e-01,1.08638033e-02,2.12669939e-01],[3.83925885e-01,-2.23253816e-02,5.30885100e-01],[-1.58332735e-01,-5.18742383e-01,-2.75313973e-01]],[[-3.58478278e-01,-1.12051718e-01,3.16549867e-01],[7.58363754e-02,-8.90821338e-01,4.99841601e-01],[-7.99656361e-02,2.65921295e-01,7.26528823e-01]],[[-4.50751632e-01,4.69386503e-02,-2.76036173e-01],[-2.42480546e-01,4.36546355e-01,-1.46265104e-01],[-7.20392098e-04,1.90339163e-01,-2.98233062e-01]],[[1.43293172e-01,8.31571579e-01,-3.71187866e-01],[-7.57795215e-01,5.27205050e-01,3.61268580e-01],[1.19893305e-01,-2.69911617e-01,1.23372388e+00]],[[2.85226852e-01,8.93032193e-01,2.43221059e-01],[-2.21936569e-01,-2.21425846e-01,3.16502340e-02],[5.46043217e-01,6.20086968e-01,-5.85923612e-01]],[[-5.90163827e-01,-1.55888885e-01,6.06425643e-01],[1.99287474e-01,4.49703515e-01,1.01383366e-01],[-4.86866921e-01,-2.73069948e-01,-1.91260278e-02]],[[5.33584714e-01,2.93271810e-01,-3.47006351e-01],[-1.30351946e-01,2.82716423e-01,1.70874655e-01],[-3.59500200e-01,4.71088551e-02,-2.77808696e-01]],[[2.24079445e-01,3.46342385e-01,-1.59201071e-01],[-3.40271652e-01,-3.92290950e-01,-2.76668936e-01],[-3.30338478e-01,7.12664783e-01,5.58264673e-01]],[[-2.18012810e-01,-8.87292624e-01,6.06182456e-01],[-3.67775351e-01,6.08428955e-01,3.10014278e-01],[-5.43750167e-01,1.37388885e-01,-1.81587070e-01]],[[1.63447596e-02,4.47143763e-01,1.35479546e+00],[-1.10433824e-01,7.83812940e-01,2.19665360e+00],[3.61769289e-01,-2.10101560e-01,-2.95734048e-01]],[[1.61091551e-01,6.87673628e-01,-1.32827044e-01],[1.65390581e-01,2.06487961e-02,4.42636460e-02],[-1.86067164e-01,-1.69081986e-02,1.07879583e-02]],[[1.26621127e-01,-5.93258560e-01,4.12886143e-01],[-3.39146346e-01,-9.06395555e-01,5.00693500e-01],[9.15566325e-01,3.23449299e-02,-1.10338700e+00]],[[1.51206538e-01,2.56557018e-01,5.59652030e-01],[3.16652924e-01,2.78091937e-01,-7.81271830e-02],[3.14339697e-01,4.18459922e-01,6.32779181e-01]],[[-3.96607608e-01,2.99060225e-01,-5.82707785e-02],[8.39034259e-01,-6.94947064e-01,2.04347834e-01],[3.32832158e-01,2.08786666e-01,-1.12460327e+00]],[[-4.00654554e-01,3.17263335e-01,-4.44527060e-01],[-8.79416943e-01,-2.06954032e-01,-1.48379475e-01],[-3.51116508e-01,2.48095706e-01,2.88904995e-01]],[[3.18220079e-01,-7.83490300e-01,1.90128803e-01],[2.75132209e-01,-5.30201308e-02,1.57976225e-01],[-2.25372225e-01,1.73413739e-01,1.65243611e-01]],[[1.10903621e+00,-9.52947363e-02,4.46794629e-01],[-2.32811615e-01,-4.95666653e-01,-5.94225168e-01],[2.10916489e-01,-3.55239630e-01,5.29989719e-01]],[[-2.12311551e-01,5.24899885e-02,-1.65034890e-01],[-3.34210306e-01,4.59272750e-02,-5.21816969e-01],[9.13552493e-02,-3.00416976e-01,-1.47058308e-01]]],[[[1.66516256e+00,-5.25256157e-01,1.43005788e-01],[4.23026025e-01,-3.86990756e-01,-3.85518491e-01],[3.85610878e-01,-2.84103483e-01,6.21798575e-01]],[[3.88741136e-01,4.88486975e-01,5.00860631e-01],[-1.72560140e-01,2.79162973e-01,-2.62980700e-01],[-2.20726535e-01,6.49173081e-01,-3.33487511e-01]],[[3.10538679e-01,4.33815420e-02,-7.26553679e-01],[1.74633682e-01,1.70709789e-01,-2.09172264e-01],[2.84496695e-01,4.26792115e-01,-3.80717307e-01]],[[5.29511034e-01,-1.78144813e-01,1.29800424e-01],[-8.46927390e-02,3.18825543e-01,-1.04504302e-01],[2.34717086e-01,-9.14429203e-02,-1.84164852e-01]],[[6.08171642e-01,1.71787977e-01,-1.28424719e-01],[4.36893344e-01,1.21363260e-01,6.20572388e-01],[-3.48855078e-01,-8.13439667e-01,6.50725842e-01]],[[-1.90107256e-01,4.82838660e-01,-2.02202052e-01],[7.38570560e-03,-1.16356919e-02,-6.57804310e-01],[2.56482810e-01,3.38966191e-01,1.00624907e+00]],[[-3.05995077e-01,-2.22032070e-01,-4.05476838e-01],[-1.49873272e-01,2.12885085e-02,-3.98932844e-02],[-2.99282730e-01,5.19390702e-01,2.63803191e-02]],[[-3.83944273e-01,-5.40936887e-01,-1.42936498e-01],[-9.46227252e-01,3.60860020e-01,-6.19545221e-01],[-6.62817895e-01,-8.09116244e-01,-5.56226015e-01]],[[7.85691857e-01,8.68467838e-02,3.00868362e-01],[-1.43991066e-02,7.72917941e-02,2.69360393e-01],[-1.93573281e-01,-3.34974766e-01,-2.09348515e-01]],[[4.06122476e-01,4.64719720e-02,1.43799081e-01],[-4.06737626e-01,-2.18612105e-01,1.77538738e-01],[-6.59053922e-01,1.68322027e-01,2.60913074e-01]],[[-4.52682227e-01,8.02032351e-01,2.90537357e-01],[-4.08093631e-01,-5.62381625e-01,1.15720645e-01],[-4.24496502e-01,-1.48826444e+00,-2.27347706e-02]],[[9.64092836e-02,-7.87349939e-02,7.90517151e-01],[-1.57775134e-02,2.42657494e-02,2.37064347e-01],[3.06655526e-01,4.80944455e-01,2.97511462e-03]],[[-1.46989688e-01,-4.34318870e-01,6.91397414e-02],[-4.12510157e-01,-4.68250364e-01,1.70350581e-01],[-1.89994052e-01,-2.34345078e-01,5.05078286e-02]],[[-3.79987322e-02,1.52862142e-03,5.23560345e-01],[3.43932003e-01,7.46419549e-01,-8.83193552e-01],[1.02481413e+00,2.51729757e-01,-9.75738227e-01]],[[1.08906627e-01,3.09846103e-01,2.86915958e-01],[-1.29126400e-01,-2.87839174e-01,3.18441272e-01],[3.45242977e-01,4.28927153e-01,2.87678808e-01]],[[-3.94636393e-01,6.30375743e-02,6.90883175e-02],[-2.72595920e-02,-1.77267775e-01,5.14683306e-01],[7.35543668e-01,-3.15143973e-01,7.35551417e-01]],[[2.59740472e-01,-3.87154013e-01,1.28169641e-01],[3.50724906e-01,3.43246132e-01,-1.59803972e-01],[-7.72040367e-01,-1.04594171e+00,-6.32561326e-01]],[[-6.08910382e-01,-2.21496478e-01,-2.22120538e-01],[8.58530104e-01,2.29364306e-01,5.35569906e-01],[1.83114961e-01,-7.36063957e-01,-1.88342124e-01]],[[-4.63700145e-01,5.88098645e-01,-1.77793875e-01],[1.81450043e-02,2.15618551e-01,-4.34206963e-01],[8.05765569e-01,-5.84930897e-01,-2.00838014e-01]],[[-5.90118527e-01,1.29692316e-01,4.51161236e-01],[-3.27756144e-02,-1.10642649e-01,-1.51413172e-01],[8.05109218e-02,2.74474621e-02,-5.82609713e-01]],[[-3.65414917e-01,4.56179291e-01,2.81836480e-01],[-4.23453212e-01,2.55290866e-01,-1.38769802e-02],[5.53472161e-01,-1.40163913e-01,3.85897122e-02]],[[1.90561593e-01,-3.97093952e-01,2.00887501e-01],[3.67337793e-01,1.69816449e-01,-3.70696843e-01],[6.48877919e-01,-1.40267804e-01,-5.85794389e-01]],[[-3.15109015e-01,-2.37996131e-02,-1.44012606e+00],[2.50894070e-01,-4.15798098e-01,-4.83473748e-01],[3.76947254e-01,-5.52884340e-02,-2.42571086e-01]],[[-9.45564628e-01,-2.52569467e-01,-1.12987548e-01],[7.55190849e-01,-4.43563312e-01,3.87094498e-01],[2.78472334e-01,7.03578144e-02,-3.01210612e-01]],[[-6.22196615e-01,-4.58077667e-03,-6.97216811e-03],[-6.21640623e-01,1.59980357e-01,-3.38116616e-01],[-4.04730774e-02,-3.08353603e-01,-2.30374426e-01]],[[4.84592438e-01,6.25153720e-01,9.68216136e-02],[-3.12842667e-01,3.39501292e-01,-2.81583905e-01],[1.78472549e-01,1.14751816e-01,9.98600781e-01]],[[1.47491962e-01,-4.47544158e-02,-2.01482743e-01],[9.02727127e-01,-2.77840406e-01,2.40125835e-01],[3.25611085e-01,1.48124406e-02,-1.16854817e-01]],[[4.08076882e-01,-1.61414787e-01,5.93805611e-01],[5.53154469e-01,2.06839889e-02,1.57173976e-01],[-3.43423903e-01,5.18607974e-01,3.46804053e-01]],[[5.83315253e-01,7.54413903e-01,1.19441390e+00],[-2.69232333e-01,-8.60448480e-01,4.60462898e-01],[5.76966777e-02,-2.89224416e-01,1.18031673e-01]],[[-9.66379583e-01,1.64078295e-01,-1.04235701e-01],[8.91006663e-02,4.64728892e-01,9.78417993e-01],[1.40204501e+00,-1.04309723e-01,-2.87570566e-01]],[[-1.45849442e+00,-1.61090717e-01,1.87694505e-01],[-3.83454323e-01,-1.03218682e-01,-1.02050193e-01],[4.37465876e-01,4.21244800e-01,4.39122558e-01]],[[-4.12573010e-01,5.68838418e-01,6.52154461e-02],[-5.37468255e-01,-6.89842701e-01,2.08022408e-02],[1.33078814e-01,-1.02141631e+00,-3.60690713e-01]],[[-7.98474371e-01,8.63424689e-02,1.08513141e+00],[2.75607765e-01,-2.52500594e-01,5.41722655e-01],[-1.15590788e-01,-2.07697481e-01,-3.69212657e-01]],[[-8.06139946e-01,-1.93902850e-01,-4.60361749e-01],[3.44590187e-01,6.71457887e-01,-2.11676657e-02],[-2.75940716e-01,5.71248710e-01,6.02797925e-01]],[[5.20185769e-01,-2.16497406e-01,6.42094076e-01],[-2.06615493e-01,2.29118705e-01,3.14013287e-02],[-3.76208723e-02,3.56750637e-01,4.58981127e-01]],[[-4.27748412e-01,4.75469092e-03,4.54160869e-01],[2.92223632e-01,-9.77177799e-01,-1.93369508e-01],[-1.48152970e-02,5.54696560e-01,9.70472813e-01]],[[-2.60279089e-01,-5.42375334e-02,1.03577577e-01],[-5.83134830e-01,-3.98642868e-01,-5.52466094e-01],[-6.75357357e-02,1.16649032e+00,2.24722639e-01]],[[2.62774944e-01,-8.48581195e-02,5.96943423e-02],[-3.54627758e-01,-8.20637941e-02,-4.76938374e-02],[-1.20670058e-01,7.42563426e-01,-1.69456631e-01]],[[1.00982118e+00,-1.51637554e-01,-1.22265548e-01],[2.22816058e-02,-8.59459519e-01,1.15720846e-01],[5.72926998e-01,-3.26495580e-02,-1.99867576e-01]],[[-1.46320835e-01,1.00991279e-01,4.58965361e-01],[-2.01795951e-01,3.02142813e-03,4.40483809e-01],[8.38872939e-02,7.37619638e-01,-2.44571306e-02]]],[[[0.06845844,-0.26245746,-0.58869964],[0.69493437,0.42478687,-0.8241714],[0.6222142,-0.40567276,-0.09918635]],[[-0.00776115,-0.38402665,0.05544258],[-0.12357549,-0.44334066,0.3477584],[0.04597204,-0.7095216,0.16646908]],[[-0.10156163,-0.07993206,0.03941975],[-0.13355121,-0.42010626,0.28903654],[-0.05262842,-0.5218527,0.50708115]],[[0.43066105,-0.32735875,0.84138733],[-0.37107027,0.2700153,0.10417134],[-0.18879302,-0.28116074,-0.26735944]],[[-0.19348113,0.23125316,-0.20840217],[-0.3058736,0.37243652,0.34620795],[-0.12153347,-0.278802,-0.47157565]],[[-0.21205835,0.34651113,-0.9187276],[-0.08565971,-0.09790248,-0.23454718],[0.5856199,-0.3858108,0.27490938]],[[-0.34257808,-0.4245439,0.36458868],[-0.53750604,0.2388643,-0.55276287],[-0.28523985,-0.10124335,-0.42630333]],[[-0.17954876,-0.03274277,0.18276647],[-0.2052548,0.12830172,-0.06621515],[-0.3338331,0.25143313,-0.10048553]],[[-0.10386129,-0.12855838,0.04230299],[0.42394617,0.34467608,-0.07521594],[-0.03696823,-0.04100119,0.63129586]],[[-0.4008612,-0.26995546,0.11898039],[-0.14611323,-0.26732108,0.19326769],[-0.57281387,-0.23365934,0.04705656]],[[0.61460716,-0.08304343,-0.1881611],[0.6639942,-0.03841714,-0.27842242],[-0.43713722,-0.36034814,-0.16189824]],[[-0.11264563,-0.04819537,0.5080767],[0.40409127,-0.04582606,-0.15235315],[0.50647175,0.11964662,-0.8764802]],[[0.11111031,0.40841988,0.0809453],[0.34581947,0.37731576,-0.30122587],[0.4921201,0.60313076,-0.5713015]],[[-1.2547184,-0.19471967,-0.3348901],[-1.5556229,-0.07619897,-0.6277566],[-1.8103118,-0.308355,-0.08517672]],[[-0.07943998,0.16210161,-0.5272027],[-0.19754732,-0.21341586,0.29764378],[0.817175,0.77090317,-0.28877905]],[[0.16049913,0.30747992,-0.30336308],[0.14258806,0.0487531,0.7968707],[-0.3937931,0.15979752,-0.48357576]],[[0.3549618,0.25285712,-0.1658878],[0.4631112,-0.7025977,-0.32601202],[0.95545864,-0.94898695,0.43463385]],[[0.24246897,0.41536638,-0.20695503],[-0.14808655,0.23229633,0.45348218],[-0.6470859,-0.12402941,-0.11437164]],[[0.24508359,-0.03531941,-0.00444494],[-0.406078,0.05808108,0.311984],[-0.27806133,-0.37609783,-0.18197607]],[[0.5822647,0.06765514,0.43895152],[0.40124223,0.42573857,0.1552642],[0.82941383,0.6895036,0.733758]],[[-0.5073925,-0.7182316,-1.3856369],[0.05972541,-0.07320591,0.20151065],[-0.83550406,-0.49807858,-0.06678951]],[[1.9255486,-0.619643,-0.01186752],[0.71453804,0.5468282,0.02887752],[0.39736477,0.04694349,0.293875]],[[0.4370094,-0.11008774,-0.18993561],[-0.05215,-0.11338998,-0.13906687],[-0.6784468,1.0422522,-0.00587705]],[[-0.34081635,0.13610528,-0.18374915],[0.0702053,0.0192829,0.35504377],[0.19025297,-0.3583768,0.3817512]],[[0.40570995,-0.14672343,0.08317091],[0.5060604,0.00217656,0.3322892],[1.1773895,-0.2438584,1.1091954]],[[-0.28683594,0.27102086,0.59651357],[0.00639111,-0.03315964,0.60364115],[0.119365,0.49961844,-0.4002046]],[[-0.01592228,0.44213182,0.11505751],[0.0484628,-0.08859291,0.35662937],[-0.82907254,-0.02797337,0.01381247]],[[0.21717049,-0.65115577,-0.06015985],[-0.01479175,0.22411406,-0.03262571],[0.44450074,-0.21982373,-0.00461884]],[[0.15242332,-0.05191997,-0.73791105],[1.1200122,-0.76390547,-1.043448],[-0.03269461,-0.30335006,-0.38794726]],[[-0.7102132,-0.5500099,0.15675716],[-1.3097366,-0.02305863,-0.43012553],[-0.25024337,-0.24581124,0.04584066]],[[-0.5966271,-0.36577612,-0.70624286],[-0.23797269,0.20540948,-0.63360274],[-0.35855395,-0.45334876,0.16642742]],[[-0.33040425,0.2832974,-0.53622866],[0.49420923,-1.0650995,-0.00874965],[0.40118936,0.5217579,0.54707015]],[[0.03473593,-0.17310512,0.06470755],[-0.01517559,-0.02125935,-0.2580648],[0.08905335,-0.2375807,-0.22008924]],[[0.65076315,0.97556096,0.11082102],[0.66556585,-0.07908983,-0.55221176],[0.2595623,-0.01770366,-0.32265416]],[[-0.38705844,0.23065357,-0.10731806],[-0.03775634,-0.08569941,-0.17193088],[-0.18088478,0.10707468,-0.7852144]],[[0.53701746,-0.1801209,-0.0865305],[0.32201806,0.17758317,-0.41208777],[-0.29709843,-0.15838106,-0.0124793]],[[0.40045977,0.40909088,-0.11378027],[-0.6475039,-0.48726162,-0.08001973],[0.26819232,0.5279282,0.14028235]],[[-0.30582932,0.3008059,-0.13406354],[-0.32329947,-0.01843113,0.88597465],[0.80054224,-0.2818649,0.13735232]],[[-0.35489064,-0.17861481,-0.24718514],[-0.1930001,0.36689103,0.49204737],[0.11829783,0.173512,0.06248983]],[[0.45921862,0.7347327,0.55029595],[-0.10208783,-0.37317932,0.26230618],[0.54916084,-0.3074066,0.31995818]]],[[[-1.32393587e+00,3.07570636e-01,2.73014009e-01],[4.13225949e-01,-4.57857639e-01,-3.12445480e-02],[-5.29673398e-01,-1.16924115e-01,3.18327963e-01]],[[9.06931683e-02,-4.70750451e-01,5.41902706e-02],[2.00069875e-01,3.67072552e-01,9.21130180e-02],[1.95166823e-02,-1.03061414e+00,1.03692997e+00]],[[1.44435191e+00,-5.14346898e-01,-3.67927432e-01],[2.47537032e-01,-1.66585863e-01,-3.97106975e-01],[1.99982990e-03,6.88529611e-01,4.98869181e-01]],[[1.93810046e-01,3.30959260e-01,-6.87762424e-02],[7.99662769e-01,-3.30304593e-01,4.28610802e-01],[1.00993648e-01,-1.02112293e+00,3.64794791e-01]],[[-9.92878497e-01,3.65034074e-01,-1.81013979e-02],[1.92255005e-01,-1.54286519e-01,3.23796779e-01],[-3.88270855e-01,-7.91978464e-02,9.06719863e-01]],[[6.31572604e-01,-1.03105657e-01,-5.56431785e-02],[1.91237479e-01,1.88325688e-01,4.30056989e-01],[7.28172719e-01,1.99670821e-01,7.35942051e-02]],[[-7.54121616e-02,-2.94047028e-01,1.86524406e-01],[2.75189370e-01,-9.15950477e-01,9.52583998e-02],[7.71344900e-01,8.47493473e-04,4.97697741e-01]],[[-1.39596328e-01,-2.12391928e-01,-1.38580874e-01],[5.98417580e-01,-2.92826474e-01,1.23773299e-01],[-2.57814199e-01,5.34862220e-01,9.06656608e-02]],[[4.55182604e-02,5.58497787e-01,-8.42430353e-01],[4.71983887e-02,-1.93885654e-01,6.18206024e-01],[4.22312140e-01,7.60231435e-01,-3.61549407e-01]],[[8.54833722e-01,1.10344701e-01,1.95480868e-01],[-2.60572910e-01,-2.47197032e-01,-6.74667060e-02],[1.45211622e-01,1.54949099e-01,3.61023962e-01]],[[-3.17198187e-01,1.91188708e-01,2.67348975e-01],[-1.53251633e-01,-4.15960312e-01,-1.80801943e-01],[2.64765739e-01,-3.79271477e-01,9.57423914e-03]],[[3.11890364e-01,-3.89591664e-01,-7.03537941e-01],[4.73315001e-01,-5.43154299e-01,-9.50009644e-01],[-1.58246264e-01,-7.04494715e-01,-1.24235570e+00]],[[5.47406599e-02,-6.56418681e-01,-7.44278014e-01],[4.61934000e-01,-6.94060847e-02,-6.21134460e-01],[9.93392542e-02,6.87230468e-01,-9.90004838e-01]],[[1.29505539e+00,-3.26531023e-01,7.28640795e-01],[1.52309775e-01,-2.66767532e-01,-5.33055007e-01],[3.08203638e-01,-4.40695584e-02,-3.15957814e-01]],[[1.79140747e-01,-4.78273600e-01,-1.52234256e-01],[1.34471118e-01,-2.47971103e-01,-3.29706162e-01],[-2.34189928e-01,4.03349310e-01,-5.49311340e-01]],[[-8.47286105e-01,2.99798846e-02,-3.55211288e-01],[1.67633638e-01,-4.39281985e-02,-6.31279290e-01],[8.00508976e-01,-1.58612266e-01,8.57516676e-02]],[[-1.48010209e-01,-1.74874574e-01,-2.37667918e-01],[5.17910682e-02,-2.46483952e-01,-3.14124435e-01],[1.06181569e-01,-5.10585010e-01,-4.68764335e-01]],[[-7.38536045e-02,1.23086475e-01,-1.37324870e-01],[-8.40603828e-01,-1.69286102e-01,-7.88824707e-02],[-2.23993152e-01,-5.67083322e-02,-3.55195224e-01]],[[4.95905936e-01,-1.14738703e+00,3.95181850e-02],[1.75382495e-01,-5.44481814e-01,-6.86661378e-02],[3.25180322e-01,-7.65202865e-02,-1.23481750e-01]],[[-2.09711224e-01,-1.67039245e-01,8.74562114e-02],[-4.16492641e-01,-4.12341684e-01,-1.38068274e-01],[3.27251822e-01,7.04051778e-02,4.13201511e-01]],[[1.52822423e+00,5.92296600e-01,8.50595236e-01],[2.77761370e-01,-2.61318922e-01,-2.71208674e-01],[1.24489339e-02,-4.04853761e-01,-3.27108741e-01]],[[-2.18987036e+00,-1.53832287e-01,-5.31406283e-01],[-1.67451632e+00,-1.65640205e-01,2.22710758e-01],[-5.41489184e-01,-5.90242505e-01,-2.24844262e-01]],[[5.95115542e-01,-5.04685566e-02,-1.04615337e-03],[1.71444595e-01,1.12620272e-01,-1.15065105e-01],[-9.24499035e-02,-6.49593547e-02,-4.17920381e-01]],[[-1.18528008e-01,-1.84446096e-01,-1.53407052e-01],[-9.59129632e-03,-2.76757240e-01,-5.16339950e-02],[-2.63194561e-01,9.11979899e-02,-1.43819362e-01]],[[-6.25143126e-02,2.63404429e-01,3.79666448e-01],[-1.84470609e-01,-7.91985154e-01,-4.25095022e-01],[-2.61635482e-02,-3.70839626e-01,-4.04061437e-01]],[[-3.76607448e-01,-5.42316079e-01,-8.92912298e-02],[2.31765985e-01,4.86591250e-01,3.05237025e-01],[1.25568533e+00,8.88698161e-01,7.14297295e-01]],[[1.12535775e-01,1.61207184e-01,3.80736023e-01],[1.31696284e-01,-1.18253514e-01,-1.92554727e-01],[2.72364676e-01,2.83297777e-01,-1.55646605e-02]],[[-2.56955940e-02,3.76678035e-02,-4.70449746e-01],[-4.67706501e-01,2.69023418e-01,-2.68119901e-01],[9.31445301e-01,-6.74447834e-01,3.14274400e-01]],[[5.85482530e-02,-1.86725035e-01,-3.03012162e-01],[3.54891956e-01,1.30810961e-01,3.32526080e-02],[-1.31374318e-02,3.08956802e-01,3.07436794e-01]],[[9.13909853e-01,2.52640963e-01,-7.01438248e-01],[-2.98050940e-01,1.32532530e-02,2.42308695e-02],[4.78598446e-01,4.83964235e-01,9.04650837e-02]],[[8.79257694e-02,5.69180429e-01,2.22731709e+00],[6.33293927e-01,-2.70324528e-01,9.28775966e-01],[-3.31107043e-02,-4.40026611e-01,-6.12483779e-03]],[[1.27153192e-02,-5.91835916e-01,9.92996320e-02],[4.32651162e-01,-3.58117878e-01,3.47988307e-01],[5.62953889e-01,-2.54046738e-01,-6.46841526e-01]],[[-1.17860667e-01,3.92418772e-01,1.42516267e+00],[-7.22983330e-02,5.40084600e-01,1.08733207e-01],[9.22809914e-02,3.04311514e-01,3.99443001e-01]],[[2.49789253e-01,-7.78571889e-02,-4.97147411e-01],[1.08549190e+00,9.88860503e-02,7.97435224e-01],[-5.75932264e-02,2.31430251e-02,-1.28516585e-01]],[[-3.84787619e-01,6.35509014e-01,3.84449810e-01],[6.45030588e-02,2.24143296e-01,3.43548328e-01],[-3.45222354e-01,1.13480210e-01,8.40826705e-02]],[[1.76290542e-01,3.98916632e-01,-9.46552306e-02],[6.33129835e-01,3.61157238e-01,3.78096104e-01],[1.24041751e-01,8.99514556e-02,-5.45072973e-01]],[[8.83193433e-01,-5.32568172e-02,-4.41894293e-01],[4.23850358e-01,2.01324120e-01,-2.36467600e-01],[9.89539921e-01,-1.42807454e-01,-2.56308019e-02]],[[3.74226809e-01,-3.01328480e-01,-1.70273893e-02],[5.12113690e-01,-4.72329438e-01,-3.03818844e-02],[2.00866088e-01,4.43032950e-01,-4.44502532e-01]],[[-5.50234616e-01,-5.58990985e-02,3.52147996e-01],[-1.98625661e-02,-1.56133249e-01,-5.14788926e-01],[6.04919553e-01,-1.71763241e-01,1.69883966e-01]],[[4.10301834e-01,5.66762865e-01,-2.96457112e-02],[-5.67043185e-01,2.27291703e-01,1.47326574e-01],[-6.68438664e-03,3.20821464e-01,-4.57059830e-01]]],[[[-1.64844489e+00,1.32292822e-01,1.03900559e-01],[-4.52344865e-01,-2.64582224e-02,-1.99187368e-01],[1.87488794e-01,5.37345171e-01,-5.62210917e-01]],[[3.01398873e-01,4.80547786e-01,4.80390161e-01],[-1.03242613e-01,1.27847299e-01,4.53406155e-01],[-2.55385190e-01,5.56483150e-01,7.74119198e-01]],[[5.83630661e-03,1.77415356e-01,-1.15346342e-01],[4.11598623e-01,4.80811694e-04,1.93414018e-01],[2.05970451e-01,2.41226271e-01,-2.54094332e-01]],[[1.35822999e+00,-2.77256101e-01,1.29237100e-01],[-3.44873555e-02,-5.44657230e-01,-4.15724397e-01],[-2.83740491e-01,2.78593063e-01,-4.62468341e-02]],[[-2.70375162e-01,-2.97245145e-01,-1.32716906e+00],[-3.54487926e-01,1.23278163e-01,-4.43757683e-01],[4.77473855e-01,-4.01179254e-01,-1.00924182e+00]],[[-1.63898602e-01,1.80025563e-01,-3.55373949e-01],[8.19811165e-01,-2.31728315e-01,-7.11384118e-01],[5.52602410e-01,-3.66937906e-01,3.96060169e-01]],[[-2.70633608e-01,-6.15592673e-02,1.39251187e-01],[-7.91913867e-01,-3.31202328e-01,-8.71582776e-02],[-7.42474794e-01,-8.54441524e-01,-7.80838847e-01]],[[-3.99889529e-01,-1.91602841e-01,6.87315822e-01],[5.68832159e-01,4.18465853e-01,5.41951239e-01],[3.39457810e-01,-1.48225069e-01,-2.74180263e-01]],[[7.27350861e-02,-4.85683203e-01,1.79813966e-01],[-2.33617812e-01,-7.53195360e-02,-1.63987488e-03],[2.67523378e-01,-8.36453140e-01,-3.91222358e-01]],[[-6.24245107e-01,5.64793088e-02,1.11754484e-01],[-5.05470216e-01,5.97540200e-01,-2.73835033e-01],[-2.93712974e-01,1.65513113e-01,6.08204491e-02]],[[-1.02867493e-02,-1.07861079e-01,6.04971275e-02],[1.41196176e-01,-1.64750993e-01,-4.51369464e-01],[-1.20967422e-02,5.78699350e-01,-1.00969172e+00]],[[-3.06442678e-01,-4.66563642e-01,9.56809446e-02],[9.96047780e-02,1.35359213e-01,5.44134617e-01],[-2.85158396e-01,-1.19650364e-01,2.39941716e-01]],[[7.19131470e-01,-5.73027790e-01,-8.09604153e-02],[2.07683325e-01,3.52885932e-01,-8.25539678e-02],[4.73723084e-01,1.46763742e-01,5.66002250e-01]],[[-9.05534148e-01,-5.10253832e-02,-1.93775713e-01],[-1.05265117e+00,1.09406337e-01,4.84239399e-01],[-9.10043895e-01,-1.81350693e-01,3.32950234e-01]],[[-4.75769937e-01,-3.08091640e-01,2.20896989e-01],[-6.51471496e-01,-4.14082587e-01,1.49363458e-01],[-3.26557726e-01,-1.58885553e-01,-5.54673016e-01]],[[-3.07922602e-01,-1.03128541e+00,-3.69242989e-02],[5.24755538e-01,4.37769204e-01,-9.40692842e-01],[-2.59583116e-01,-1.16360923e-02,-5.14597297e-01]],[[1.33057600e-02,-2.28624135e-01,6.78517818e-01],[-1.01613864e-01,-4.60552037e-01,4.41616386e-01],[1.26222372e+00,-6.43982172e-01,-6.81803450e-02]],[[-8.35475266e-01,3.33338141e-01,1.84077680e-01],[1.83802068e-01,4.27352190e-01,2.71526933e-01],[1.07338347e-01,-2.96598703e-01,2.84013420e-01]],[[2.71525979e-01,-1.07074738e-03,-1.70716539e-01],[2.10150912e-01,5.10786235e-01,1.25016049e-01],[1.09133625e+00,-8.49886239e-01,-2.76017010e-01]],[[-3.34887505e-01,-9.52277035e-02,-8.10513437e-01],[-3.89871746e-01,-3.23563159e-01,-4.99033004e-01],[7.15586603e-01,4.08538654e-02,1.05051482e+00]],[[2.54287515e-02,2.67979473e-01,3.65535021e-01],[-9.66334522e-01,-4.64375556e-01,-2.57133693e-01],[-7.20914304e-01,-1.00374505e-01,-2.22875196e-02]],[[1.02187097e-01,5.07781208e-02,5.69217622e-01],[-3.37969750e-01,2.30047792e-01,-2.50566751e-02],[-3.77098501e-01,-7.18645751e-01,-3.57566535e-01]],[[-1.14690758e-01,-1.24441795e-01,3.45867157e-01],[-1.08409271e-01,-5.88586926e-02,1.04881763e+00],[9.60262716e-02,2.65509009e-01,5.08630693e-01]],[[-7.20743477e-01,1.25495404e-01,-6.88863024e-02],[-7.97019601e-01,-1.78631902e-01,-8.63884240e-02],[-6.77679479e-02,4.08416718e-01,2.51824349e-01]],[[-3.40563923e-01,3.71587664e-01,4.02333468e-01],[-1.41111523e-01,-3.04923862e-01,-7.39450932e-01],[-1.18732817e-01,3.05187865e-03,2.56773055e-01]],[[5.59414983e-01,2.81380564e-01,-3.43012899e-01],[2.63354599e-01,-2.27465317e-01,-7.04138815e-01],[-5.73322594e-01,-6.55723989e-01,-5.49421251e-01]],[[-2.33859733e-01,3.60357463e-02,-5.88115811e-01],[-6.09053224e-02,1.73951909e-01,-7.30588734e-01],[-1.40987024e-01,-3.88126411e-02,6.75162673e-02]],[[1.27623662e-01,7.80516088e-01,1.00984082e-01],[-1.40510008e-01,1.24311365e-01,6.70155048e-01],[9.30033863e-01,-5.22755682e-01,4.97145914e-02]],[[-1.39881477e-01,-2.37818375e-01,-7.82409191e-01],[-2.36932740e-01,-6.06260955e-01,1.24015316e-01],[-3.64126384e-01,-2.57468224e-01,3.92655998e-01]],[[-3.30585867e-01,-1.29863530e-01,2.08105043e-01],[-9.84409869e-01,-1.02583599e+00,1.52373806e-01],[4.51764077e-01,-4.61544722e-01,-2.14669202e-02]],[[-5.53921282e-01,-5.01101315e-01,1.09009363e-01],[1.71091467e-01,3.22609365e-01,2.70674616e-01],[-2.63939500e-01,2.59357631e-01,2.64720738e-01]],[[-9.06055719e-02,1.07988723e-01,1.80431470e-01],[1.05573666e+00,-3.85293096e-01,6.59139812e-01],[-9.41273689e-01,-6.63088441e-01,-9.62723121e-02]],[[-4.45328712e-01,2.39157483e-01,-2.22102463e-01],[-9.99800712e-02,-4.69976783e-01,1.88823909e-01],[-9.96218026e-02,1.97982192e-01,-1.07993022e-01]],[[1.01066507e-01,-1.79332808e-01,-6.85089603e-02],[3.30725431e-01,-1.34578601e-01,1.13572523e-01],[4.02564794e-01,2.78157234e-01,-3.06251258e-01]],[[-2.19406605e-01,-6.44294918e-01,-1.82622507e-01],[6.76147997e-01,-1.15253277e-01,-9.83594835e-01],[4.66009118e-02,-6.89809442e-01,-1.02804935e+00]],[[2.77998418e-01,2.36502782e-01,-4.58853543e-02],[-2.91326523e-01,6.00765586e-01,-4.39944655e-01],[1.34663910e-01,6.62328839e-01,-4.61804509e-01]],[[4.93354738e-01,7.89742544e-02,-8.26704949e-02],[1.59584612e-01,4.87524837e-01,-6.04930043e-01],[2.66692460e-01,1.17123593e-04,-2.93113440e-01]],[[1.57963574e-01,1.36389837e-01,-1.89506918e-01],[-5.31548977e-01,-3.15087885e-01,-9.68624428e-02],[7.06213415e-01,-3.98670614e-01,3.04999739e-01]],[[-7.63747215e-01,-3.66000365e-03,5.07926762e-01],[-1.00161541e+00,4.16841030e-01,-2.20727488e-01],[-2.91929781e-01,1.69334963e-01,2.00043127e-01]],[[3.30708116e-01,-7.53446296e-02,3.97949010e-01],[2.71316975e-01,3.10481548e-01,-1.46268189e-01],[-3.06159824e-01,2.63573498e-01,-2.84348369e-01]]],[[[3.80441606e-01,5.69501705e-02,2.96693206e-01],[1.47949800e-01,-3.93503338e-01,-1.09589243e+00],[3.80076095e-02,-2.44201571e-02,2.81872153e-01]],[[2.12111950e-01,-2.38077879e-01,-7.63293028e-01],[1.16718674e+00,-8.19056407e-02,1.31190017e-01],[5.59645712e-01,4.14608806e-01,-2.81064570e-01]],[[6.36756897e-01,2.18623683e-01,2.62113601e-01],[-2.67464250e-01,-1.37255654e-01,2.38290861e-01],[1.88652426e-01,-1.29079744e-01,5.62085330e-01]],[[3.06305945e-01,3.86438876e-01,-8.77217054e-02],[-8.05038214e-02,-5.00491619e-01,2.27857426e-01],[-5.47170192e-02,5.73794603e-01,6.04203820e-01]],[[3.98959666e-01,9.78229463e-01,-1.86171949e-01],[-1.03810918e+00,7.28598118e-01,-4.10599053e-01],[-2.78102830e-02,-7.05038607e-02,-8.05426389e-02]],[[-2.37980559e-01,-2.85070240e-01,1.57083258e-01],[-8.46953630e-01,-3.34712029e-01,-2.35328794e-01],[-3.83400023e-01,-2.09692240e-01,-3.18419605e-01]],[[6.41149580e-01,1.01929963e-01,-1.78319529e-01],[3.85943919e-01,3.03169824e-02,-1.48483321e-01],[1.59990937e-01,-3.73853594e-01,1.14789821e-01]],[[-9.93174016e-01,-3.46321128e-02,6.87981620e-02],[-5.52467942e-01,-3.63844000e-02,-5.91893792e-01],[9.17671502e-01,6.62158668e-01,-4.65774596e-01]],[[-4.00107093e-02,-1.15027440e+00,3.19491416e-01],[-4.43253636e-01,-5.71952581e-01,3.03799748e-01],[-4.73580748e-01,4.06233519e-01,7.36669123e-01]],[[-3.12087059e-01,-1.40490383e-01,1.66705679e-02],[-1.40733182e-01,-6.23066783e-01,1.46632433e-01],[1.86584175e-01,-1.90968573e-01,-8.90614033e-01]],[[-1.60881087e-01,-6.97104931e-01,-7.40375146e-02],[-2.44057044e-01,3.15623969e-01,1.40451014e-01],[-8.71666431e-01,-4.69574690e-01,2.35141024e-01]],[[-8.00852239e-01,8.50661397e-01,-5.78055203e-01],[-3.07636186e-02,1.12378500e-01,-4.88829225e-01],[-7.05498755e-01,6.57415390e-01,-1.70083478e-01]],[[4.97484118e-01,-1.06080778e-01,3.21173042e-01],[1.26478210e-01,9.93747711e-02,3.67403030e-01],[-3.54541302e-01,-5.57190299e-01,-5.34188449e-01]],[[2.92447448e-01,2.27849800e-02,2.65571862e-01],[3.92445862e-01,-4.10987288e-01,-2.99090654e-01],[8.60455215e-01,-4.81634498e-01,2.97013432e-01]],[[2.99379230e-01,4.84369189e-01,2.06499398e-01],[3.94242071e-02,4.73902494e-01,-5.93104124e-01],[-5.70094228e-01,1.26325101e-01,-4.01445836e-01]],[[-5.77260852e-01,3.10068935e-01,5.04750669e-01],[1.51412889e-01,-1.51956812e-01,-4.49412614e-01],[3.74922812e-01,9.82664764e-01,-5.76092184e-01]],[[-2.67235160e-01,3.54707956e-01,2.65998840e-01],[-7.28119984e-02,2.44537964e-01,-3.14906329e-01],[-8.02925229e-01,-2.38997847e-01,7.38764778e-02]],[[4.40120906e-01,2.99495369e-01,-6.40579313e-02],[3.38428438e-01,-3.49413827e-02,6.53561831e-01],[-1.09809991e-02,1.35731190e-01,-5.84280133e-01]],[[4.88931328e-01,2.68010825e-01,-2.24258587e-01],[6.12001717e-01,-2.23792300e-01,-6.69959664e-01],[3.56896594e-03,-6.37818694e-01,1.69689879e-01]],[[1.23939492e-01,-7.73222983e-01,-4.10224944e-01],[-3.36989701e-01,-2.03487918e-01,1.05756152e+00],[-1.27074444e+00,-5.52473783e-01,-2.24502936e-01]],[[2.63549685e-01,-3.61418188e-01,-9.99710023e-01],[1.25757337e-01,2.32352257e-01,-6.58488810e-01],[1.85669258e-01,-5.14040053e-01,-1.07233548e+00]],[[-1.84024662e-01,3.98879945e-01,-4.07790393e-02],[-6.83365762e-01,6.20326936e-01,2.24523768e-01],[3.62946749e-01,4.04764354e-01,6.75883770e-01]],[[-2.43569687e-01,-9.17267084e-01,-1.73737913e-01],[-1.09650850e-01,1.79257318e-02,-4.87618953e-01],[5.32168746e-02,2.34734043e-01,1.09452292e-01]],[[-3.42119746e-02,-2.89164901e-01,-6.67871356e-01],[1.93268210e-01,2.70747393e-01,2.43558973e-01],[-1.31859377e-01,6.75231814e-02,1.58795953e-01]],[[-7.87870944e-01,-4.56544816e-01,-7.15385914e-01],[6.11032486e-01,7.72035956e-01,-3.76177639e-01],[1.40574381e-01,-6.83917403e-01,3.65055442e-01]],[[-6.38759077e-01,-3.36072147e-01,6.60158217e-01],[-2.01930866e-01,8.06258678e-01,5.38107634e-01],[3.24091688e-02,-6.10421717e-01,-1.52343228e-01]],[[-4.53126252e-01,1.50805667e-01,-1.41776606e-01],[4.29449022e-01,-2.58033842e-01,5.12962401e-01],[-2.27935970e-01,-3.99989128e-01,1.03341021e-01]],[[6.64515495e-02,-4.65764523e-01,-4.66931164e-02],[-6.18111968e-01,-1.96549684e-01,-1.44801155e-01],[-1.17249899e-01,1.13878286e+00,6.54089153e-02]],[[-3.69528472e-01,-2.92739838e-01,3.19075704e-01],[5.56756146e-02,-4.05820400e-01,2.48137563e-01],[-3.24688643e-01,3.74416173e-01,-2.84630150e-01]],[[4.19900753e-02,-3.96830738e-01,-4.05565560e-01],[5.25885642e-01,-5.40038705e-01,-5.54797053e-03],[6.07051909e-01,-4.12532330e-01,3.67172360e-01]],[[7.19066620e-01,-1.98122621e-01,1.02341063e-01],[6.09969854e-01,-3.51750404e-01,-1.95746928e-01],[3.90545607e-01,-2.60870546e-01,-2.04644337e-01]],[[-5.94774112e-02,5.10767341e-01,1.19174860e-01],[1.47410437e-01,6.31260991e-01,6.75190240e-02],[4.96671855e-01,5.45517623e-01,2.98308790e-01]],[[5.44846356e-01,4.36644942e-01,6.78651512e-01],[-3.87868226e-01,1.63454577e-01,-3.34241956e-01],[1.91421494e-01,-1.03635876e-03,-6.43176064e-02]],[[3.16162288e-01,-5.00748575e-01,3.69972169e-01],[-6.16354756e-02,-9.24472868e-01,5.29171526e-02],[-4.04741973e-01,-1.22793525e-01,-1.34140134e+00]],[[2.19535381e-01,2.55529940e-01,1.09388387e+00],[1.46886393e-01,1.89300582e-01,1.62590206e-01],[2.60164171e-01,7.40840256e-01,-2.03629211e-01]],[[2.75333464e-01,1.56320274e-01,-6.25223339e-01],[-4.02452916e-01,2.71440476e-01,-3.08129769e-02],[-9.36767280e-01,6.32205784e-01,-1.79187208e-01]],[[6.49394989e-01,1.37401432e-01,3.36037017e-02],[-4.88460302e-01,-5.42944551e-01,-5.04809618e-01],[-5.11417150e-01,-5.29914320e-01,-1.81253955e-01]],[[-7.15729296e-01,3.67578030e-01,1.68161497e-01],[3.62656146e-01,3.71582210e-01,4.03503180e-01],[1.86496269e-04,-1.76406935e-01,-6.93188608e-02]],[[6.74176037e-01,-4.60460991e-01,-3.41528535e-01],[-5.71862936e-01,-2.22267300e-01,-4.89119798e-01],[-2.27836341e-01,-1.01477265e-01,1.90480158e-01]],[[8.88846517e-02,2.13885054e-01,-3.34741324e-01],[-1.03070140e+00,3.71785849e-01,2.20319912e-01],[-1.02612078e+00,2.38681242e-01,-3.04297209e-02]]],[[[1.86428115e-01,-4.26940806e-02,8.69498491e-01],[1.20639116e-01,6.95010781e-01,3.48774433e-01],[-1.46759108e-01,-9.48503315e-01,-3.25774759e-01]],[[4.46270585e-01,1.54580891e-01,-1.21244781e-01],[-3.01690362e-02,1.92099005e-01,-6.12704992e-01],[-3.44512939e-01,8.93007517e-01,5.41917980e-01]],[[3.63023072e-01,1.69253513e-01,-7.97942460e-01],[3.16798240e-01,-5.11660725e-02,-1.43924549e-01],[-6.54457867e-01,1.55161275e-02,8.78646504e-04]],[[2.01898068e-01,5.46159029e-01,-3.01666409e-01],[-4.66165900e-01,-5.64843833e-01,4.45906669e-01],[-2.49043182e-01,-1.74204946e-01,-7.99965739e-01]],[[-3.73042405e-01,3.02940428e-01,-1.07310683e-01],[-9.29876491e-02,-1.37502655e-01,-2.60073423e-01],[-7.41362691e-01,-1.01207304e+00,3.78707677e-01]],[[6.53745770e-01,-1.43314466e-01,3.38020734e-02],[1.08713591e+00,1.02976672e-01,-2.20974624e-01],[-6.10165119e-01,-3.14227670e-01,-6.27702832e-01]],[[-1.94050848e-01,-7.95293003e-02,2.95619339e-01],[2.63999924e-02,6.07307255e-01,-2.61390060e-01],[-8.35823655e-01,-9.91545767e-02,3.24362338e-01]],[[6.27198875e-01,-7.05508143e-02,-2.11121142e-02],[7.99368858e-01,-3.03679407e-01,-2.43586466e-01],[1.52087519e-02,-2.08904982e-01,3.03318679e-01]],[[-2.33211949e-01,-2.19534084e-01,-8.23127687e-01],[9.13300514e-02,6.54377699e-01,-2.21975908e-01],[-8.90587747e-01,-6.37088045e-02,-3.35183889e-01]],[[-5.29469773e-02,9.54655528e-01,-1.07571363e-01],[-7.69564658e-02,8.28010678e-01,5.26589394e-01],[4.13120359e-01,-7.01982915e-01,3.71895842e-02]],[[1.74851287e-02,2.28495896e-01,8.71028304e-01],[-3.21246892e-01,2.07524329e-01,7.08433390e-02],[1.17689691e-01,-6.98549569e-01,-5.83161175e-01]],[[-2.90967107e-01,1.31705418e-01,-9.23693895e-01],[-1.55869976e-01,-1.04922317e-01,5.26207983e-01],[4.19081211e-01,-7.32954383e-01,5.35943955e-02]],[[-1.55184701e-01,-5.57147324e-01,9.39638242e-02],[2.94059575e-01,-2.95612395e-01,3.02756935e-01],[-5.76877832e-01,-5.13434820e-02,-2.48953626e-01]],[[5.35656393e-01,6.08243048e-01,3.90167028e-01],[-4.83726978e-01,-1.51033215e-02,9.99266684e-01],[-1.96421877e-01,9.62535024e-01,-3.72289643e-02]],[[-1.80482104e-01,1.08383201e-01,-3.94284487e-01],[-4.50743884e-01,5.66799462e-01,-4.81785208e-01],[-1.60538709e+00,4.49380547e-01,3.82829607e-01]],[[6.48413450e-02,-4.79413122e-01,1.91715986e-01],[2.39918515e-01,1.16286883e-02,4.79252219e-01],[-4.74258810e-01,-7.21511662e-01,-6.60528541e-01]],[[1.95885628e-01,-3.32274318e-01,4.03654948e-02],[-1.19572639e-01,1.93918362e-01,8.94486308e-02],[4.88810614e-02,-3.22482497e-01,8.55205119e-01]],[[-1.88853815e-01,-2.89775524e-02,-3.21183920e-01],[-8.26217055e-01,-1.04905939e+00,6.91808835e-02],[-2.29646564e-01,3.96897763e-01,3.99507523e-01]],[[4.99365240e-01,-1.23534784e-01,-7.69860327e-01],[4.71358716e-01,-2.23125115e-01,2.96360664e-02],[4.24844772e-02,2.48172939e-01,-3.19116116e-01]],[[-2.38834918e-01,-8.54608953e-01,-1.12246168e+00],[-8.95671308e-01,-2.49978527e-01,-6.58918694e-02],[-8.66254032e-01,1.35411203e-01,-5.13583779e-01]],[[-1.27754156e-02,1.18854865e-01,1.70334411e+00],[1.34890497e-01,8.36162567e-01,7.17781544e-01],[2.97452956e-01,4.10237879e-01,-1.30622536e-01]],[[2.61228770e-01,-1.26628458e-01,-4.76677895e-01],[1.00443698e-01,4.61601585e-01,-3.75440866e-01],[1.82835892e-01,2.05673784e-01,9.33449790e-02]],[[1.49151469e-02,3.19101304e-01,-7.95187116e-01],[2.34024953e-02,-4.55539137e-01,-4.97521847e-01],[1.51327237e-01,1.38733611e-01,-2.76782334e-01]],[[2.43148014e-01,1.82002664e-01,-5.09871125e-01],[-1.89585686e-01,4.57662940e-01,-1.76652238e-01],[9.46699679e-02,-1.37483209e-01,-2.53810793e-01]],[[-1.31958172e-01,-4.13115382e-01,-4.82399106e-01],[6.44996911e-02,2.44579926e-01,-4.33542073e-01],[5.66364944e-01,-5.47538877e-01,6.80826724e-01]],[[5.44686556e-01,3.99017483e-01,-5.11184812e-01],[2.37096980e-01,-1.47481393e-02,7.71569684e-02],[-4.44111377e-01,-5.70451319e-01,9.72526073e-01]],[[4.96646911e-02,6.42541885e-01,-5.48615865e-02],[8.34829882e-02,-2.45000243e-01,4.38110590e-01],[-2.74840370e-02,-3.42420250e-01,5.07013425e-02]],[[-1.47428498e-01,-1.80108160e-01,-1.83016121e-01],[8.22252631e-01,2.26244628e-01,2.22101465e-01],[-2.54574627e-01,-4.76949513e-01,6.30077943e-02]],[[-5.58516622e-01,-2.88356215e-01,6.51652873e-01],[4.42362219e-01,2.05149367e-01,-1.99351072e-01],[-8.68793428e-02,-4.95918453e-01,1.40062034e-01]],[[5.59737347e-02,-1.16152719e-01,-1.62173435e-01],[2.25718230e-01,5.80871284e-01,-3.64797145e-01],[4.81590629e-02,2.09058613e-01,3.99435431e-01]],[[-4.17348832e-01,4.08421695e-01,1.63455832e+00],[-8.21027517e-01,-9.65309069e-02,8.24752212e-01],[-1.12118371e-01,3.56260575e-02,5.01369655e-01]],[[-6.60968721e-01,2.45642006e-01,-1.96825236e-01],[-7.56884158e-01,4.93323393e-02,-4.75609899e-01],[-5.58092713e-01,-1.11125040e+00,-2.31556907e-01]],[[-7.38042444e-02,8.61579657e-01,1.20915484e+00],[-7.67557919e-02,7.31446385e-01,5.44433892e-01],[-1.46994427e-01,1.69469733e-02,-6.59579575e-01]],[[2.34289691e-01,5.47921844e-02,-2.48599887e-01],[-1.02435336e-01,-3.48358750e-01,-2.05801412e-01],[3.78883272e-01,7.04093277e-01,5.36094010e-01]],[[3.12749058e-01,3.07786148e-02,-2.00728830e-02],[2.06341460e-01,-6.68197684e-03,1.44843549e-01],[-2.93383151e-01,-2.98241049e-01,-5.66247523e-01]],[[5.23429662e-02,3.14217150e-01,4.23386276e-01],[6.77726150e-01,-1.25026584e-01,-4.03794736e-01],[-8.49415064e-02,2.62758553e-01,4.66049403e-01]],[[3.81274894e-02,-3.26051831e-01,-5.66853166e-01],[-2.02305496e-01,-5.57965159e-01,1.53958842e-01],[1.50041804e-01,4.62690413e-01,-3.61652635e-02]],[[-4.92287785e-01,4.58025545e-01,-8.95657837e-02],[-3.71642262e-01,5.15328646e-01,4.99197125e-01],[-6.73084036e-02,-7.10377991e-01,-4.43896741e-01]],[[-8.45360607e-02,1.18462357e-03,-1.73649326e-01],[5.63504398e-01,-4.66855109e-01,1.96251675e-01],[2.17134058e-01,-6.01326644e-01,-2.22534820e-01]],[[7.16417909e-01,-2.70867229e-01,2.37479299e-01],[7.95698106e-01,-1.93636179e-01,4.60170209e-02],[1.05623230e-01,-4.44124967e-01,6.85597301e-01]]],[[[0.21635437,-0.20578022,-0.08655912],[0.36080605,-0.1972738,0.58030015],[-0.46212465,0.37218416,-0.5816711]],[[-0.1901254,0.9099191,-0.3260427],[0.5880226,0.33582017,0.40765953],[-0.25787818,0.52910465,-0.48435554]],[[-0.05438785,0.30038658,0.31371382],[0.40977675,0.5828006,-0.5412878],[-0.27519986,-0.13800725,0.41178057]],[[0.08964548,-0.25553107,0.4097002],[0.1949622,-0.56196576,-0.7924259],[-0.54147977,0.31485087,-0.10666711]],[[-0.79263276,0.21982248,0.36731336],[0.56028175,-1.0002286,0.3067577],[-0.29264176,-0.12850687,-0.1604451]],[[-0.39336815,0.5475028,0.8343868],[0.9676911,-0.22827446,0.29117668],[0.14706743,0.4047691,0.00221942]],[[-0.30479842,0.35706282,-1.2002578],[-0.36322236,-0.13544591,0.09118361],[0.7899216,0.31944776,-0.59210455]],[[-0.00410059,0.5046418,0.26479605],[0.6830972,-0.10409639,0.9194381],[0.00164416,-0.2000344,0.7322063]],[[-0.78078467,-0.4064936,0.37123907],[0.2973654,-0.7269893,-0.01299352],[0.2123233,-0.42802066,-0.5812783]],[[-0.5541974,0.21552512,0.15194239],[0.3328718,-0.66750777,0.46117443],[-0.12184343,0.37104717,-0.18438329]],[[0.7479842,-0.48729697,-0.6971616],[-0.3083951,-0.38408622,-0.68567455],[0.12064114,0.20010659,0.12394427]],[[0.83895665,0.60365885,-0.28730717],[0.64449686,-0.10431446,-0.01954602],[0.03628849,-0.4217415,-0.6107948]],[[-0.18002668,0.10936163,0.16654342],[0.8223783,-0.23572251,0.5992473],[0.12690198,-0.5335599,0.32079184]],[[-0.12725928,0.34015733,0.29394644],[-0.5037057,0.09869266,-0.4155538],[-0.24660563,0.3095755,-0.00647587]],[[0.49394372,-0.07423196,0.4507026],[-0.44197175,-0.381439,-1.3662276],[-0.04144278,0.52486813,-0.0201037]],[[0.6005469,0.1926683,-0.40764531],[0.07151769,0.22529681,-0.71356547],[0.5367786,0.4353252,0.11172152]],[[0.02776276,0.23011985,-0.59416384],[0.16122547,-0.20148787,-0.03654109],[-0.7010277,-0.09827253,-0.3627937]],[[-0.25441217,0.15431102,-0.5786317],[0.07456255,0.01730599,0.7152011],[0.21934818,-0.08463073,-0.40247408]],[[0.3113152,-0.37166366,-0.5256368],[-0.38448718,-0.09938992,0.3852833],[-0.43015322,0.38194287,0.6590988]],[[-0.03160679,-0.27865595,-0.9036851],[-0.49477282,0.07965773,-0.40519726],[0.2624988,-0.28714865,0.00953154]],[[-0.36116588,0.61610526,0.897853],[-0.16107658,0.8164856,-0.10107309],[-0.18775085,0.45280284,0.20098849]],[[0.49573964,0.25051543,-0.17471334],[0.08592154,0.58649755,-0.0321577],[0.02128373,-0.302959,0.13267474]],[[0.27809018,0.70029414,-0.7878594],[-0.9465564,0.15340063,-0.14373255],[0.05656678,-0.349642,-0.9353123]],[[0.08170329,-0.20203906,-0.306505],[-0.6394185,-0.36734018,-0.7835165],[0.3181892,-0.34643734,-0.6608513]],[[-0.47749543,-0.2812454,0.08318888],[-0.25533354,0.03386112,0.32794654],[-0.4394104,0.44730866,-0.9512937]],[[0.3847846,0.6252628,-0.34890205],[-0.5709454,-0.6135237,-0.36099416],[1.2552269,-0.21077523,-0.8764651]],[[0.10753749,-0.3300722,0.03292734],[0.03644223,0.39515194,-0.11137915],[0.22329672,0.7023171,-0.41291362]],[[0.18048374,-0.4771267,-0.40021124],[0.6655195,-0.6920188,-0.6236417],[-0.42512387,-0.00697205,0.39526948]],[[-0.15629224,-0.5815554,-0.44744107],[-0.48559758,-0.08149138,-0.53376234],[-0.31585902,-0.26171845,0.5219889]],[[-0.39424974,-0.12186718,0.1948737],[-0.46889204,0.06816499,-0.05319411],[0.78367144,-0.3190728,0.26018545]],[[0.14035107,0.05447598,-0.45827985],[-0.3330277,0.46645904,-0.19660418],[-0.6826136,-0.68701714,0.00308301]],[[-0.14954026,0.53986937,-0.16180277],[-0.20429035,-1.2524971,-0.1235166],[-0.12773213,-0.04531156,-0.94002306]],[[-0.4485285,-0.14439344,-0.23468553],[0.3129257,-0.15066661,-0.3947746],[-0.45969653,-0.33370087,0.27134472]],[[-0.47196323,-0.20910093,-0.5280208],[-0.8254305,0.23874806,-0.03284088],[-0.04914233,-0.09368979,0.28326893]],[[0.08943992,-0.5590408,-0.62069553],[-0.37803262,0.00653033,-0.569026],[-0.196984,0.32263803,0.08200127]],[[-0.01280908,0.6753974,-0.18599673],[-0.2905319,-0.30975118,0.05893749],[-0.24325573,0.37475434,0.02150218]],[[0.73543656,1.0024635,-0.11627571],[-0.5957181,-0.0431121,-0.19374792],[-0.19092557,0.69684297,-0.1962828]],[[1.1264564,0.39092305,0.22857384],[0.12761745,0.77118474,0.17684782],[-0.36876753,-0.02340429,0.66141886]],[[0.91579366,0.77191377,-0.47071943],[0.18897605,0.61446285,0.57172084],[-0.23128483,0.07578982,0.82605225]],[[0.40921187,0.57280785,0.4381099],[0.21064039,0.32505792,0.66649234],[-0.6521096,0.28606972,0.11794586]]],[[[0.640884,0.88570964,0.47792855],[0.09590463,0.03423553,-0.02406613],[-0.9782734,0.34997895,-0.12867458]],[[0.4731407,-0.02430954,-0.32651138],[-0.45940375,-0.30068043,-0.05939324],[0.01173439,-0.6792217,-0.29455352]],[[0.28921524,-0.55465627,0.5767435],[0.08637864,-0.1631998,-0.2816488],[0.04125011,-0.47419667,-0.49502444]],[[0.06713139,0.21520804,-0.61054426],[0.392247,-0.39424956,-0.24268028],[0.48073873,0.18782428,-0.11485723]],[[0.4458465,-0.5916154,0.55501133],[0.48593682,-0.19297457,0.8720179],[-0.05490496,-0.05922987,1.419253]],[[-0.21467721,0.27886683,0.11617681],[-0.30673358,-0.3739717,-0.73872966],[-2.0139394,-0.68753505,-0.617372]],[[-0.5283664,0.20439926,0.13677566],[0.15618339,-0.27368602,0.49916476],[-0.1883965,-0.01264808,0.2839805]],[[-0.15147378,-0.7269368,-0.08356918],[0.16574731,-0.5677674,0.29295647],[0.14449787,-0.618238,-0.11977262]],[[-0.23333336,0.2359647,-0.4606852],[-0.02785524,0.4431884,-0.73755026],[-0.48673195,-0.22752114,-0.61578745]],[[-0.14242752,-0.12764539,-0.15173505],[-0.628586,0.38559192,-0.21085529],[0.18547872,0.30963048,-0.33239576]],[[0.10762382,-0.4156635,-0.05268024],[-0.09254022,-0.5526826,0.46000913],[0.40828717,-0.5389552,0.02427388]],[[0.6376271,-0.1427635,-0.22221886],[0.74527395,0.30545056,0.02705691],[-0.21189088,-0.15384923,0.02996311]],[[0.0981339,-0.16115382,0.35083044],[-0.48696265,-0.18443114,0.11819646],[0.08438414,-0.8927129,0.25942808]],[[0.01040233,0.37014893,-0.17146495],[1.1580193,0.10658061,-0.02077313],[0.96432394,0.38714728,0.44040197]],[[0.5318611,0.3164958,-0.40529886],[0.04714479,0.39541003,0.29506323],[-0.24708627,-0.3095797,0.42669517]],[[-0.07213557,-0.36890125,0.2504379],[0.11784945,-0.40625077,0.478494],[0.01703687,0.16346426,0.53454775]],[[0.15285014,-0.3751912,0.26597434],[-1.1854571,-0.07436091,0.7258832],[-0.9627232,-0.15344696,0.25412104]],[[0.1759837,-0.15960261,0.89562315],[0.1975376,-0.25156975,-0.02686702],[-0.11581048,0.03333941,-0.21428357]],[[-0.42281628,0.3925426,-0.08151293],[-0.13048945,-0.659519,-0.56547445],[-1.0415257,0.6298423,-0.4834254]],[[-0.26906127,-0.10125737,0.39759576],[-0.07902454,0.4971146,-0.10619045],[-0.66157687,0.28331664,-0.7126975]],[[0.14766942,-0.2714179,0.65739137],[0.00768742,-0.32113674,-0.02514714],[0.05805905,0.20633142,0.5410461]],[[-0.14285322,0.07050283,0.40880078],[0.4900249,-0.03171129,0.26089984],[0.2613414,0.01013389,0.22517431]],[[-0.4885949,0.11236713,-1.6635174],[0.5629332,-0.10452516,-0.10651257],[0.2869399,-0.37566763,-0.8250944]],[[-0.32525977,0.25571698,-0.8765356],[0.44977674,0.5270927,0.5947032],[-0.45119396,0.22459157,-0.6737102]],[[-0.04339676,-0.5115286,-0.5976126],[-0.35669243,0.6882545,-0.13934818],[-0.38795948,0.64660215,-0.63246065]],[[-0.25516626,-0.49955443,-0.14739846],[-0.64446485,-0.25205278,0.0255373],[-0.06866008,-0.3798844,-0.09173334]],[[0.36808333,-0.31512168,-0.6328398],[0.15770109,0.10491087,-0.37336934],[-0.32070187,0.02485435,-0.3050682]],[[0.49403483,-0.27764833,0.05249931],[0.6160992,-0.3933422,-0.8726022],[-0.45976928,0.02184594,-0.3779153]],[[0.3868703,0.02338008,1.4441525],[-0.13729632,0.41938943,0.69478244],[-0.39154202,-0.25218612,0.83274686]],[[0.22876543,0.23255679,-0.07165794],[0.4789886,-0.45760393,0.15989916],[1.103498,0.06196279,0.84567547]],[[0.02679596,0.7460273,0.3826153],[-0.22714162,0.545457,0.21949992],[0.56237465,-0.29980433,0.43944344]],[[-0.18409297,0.09194113,0.17096949],[0.22896044,-0.54051846,0.14188136],[0.2886905,0.20861845,-0.39245215]],[[-0.5433605,-0.14361858,-0.28816175],[-0.7944679,0.4374666,-0.31361252],[-0.01914958,-0.13739866,-0.20930803]],[[0.41229665,0.15081666,0.54791266],[0.22188385,-0.01271763,0.98311013],[0.82176906,0.5157679,0.75915617]],[[0.19377825,0.38373655,-0.0384186],[-0.24156316,-0.3239602,0.29110938],[0.38279477,-0.15942232,0.900104]],[[-0.28647405,0.19768749,0.29259714],[0.26922876,0.1754741,-0.29270983],[-0.81087667,-0.0657342,-0.03151231]],[[0.01108314,0.90632474,0.29444933],[-0.17814182,0.18045004,0.51246953],[-0.30907908,0.67577845,0.55539876]],[[-0.24655528,0.16046768,-0.14278845],[-0.1886212,-0.09743374,-0.05214272],[-1.4128453,-0.5368601,-0.6321441]],[[0.25231323,0.4200192,-0.22554854],[0.16977818,-0.08192085,-0.07054517],[0.2964064,0.4355419,-0.2908669]],[[-0.37476006,0.2238652,-0.04184174],[-0.49738738,-0.2576274,0.34601343],[-0.3100724,-0.7204148,0.2822738]]],[[[6.42177165e-01,3.50112557e-01,-6.38169646e-01],[-2.47081399e-01,2.87749141e-01,1.51841357e-01],[-3.96605760e-01,-3.05634618e-01,-6.07944310e-01]],[[6.64235592e-01,-2.14587465e-01,-8.27796936e-01],[8.00652683e-01,6.51735187e-01,2.32743114e-01],[-8.95799518e-01,1.24627948e-01,6.73089087e-01]],[[-7.12175965e-02,-3.73300612e-01,1.11827219e+00],[2.24043056e-01,-3.53677087e-02,1.48738414e-01],[-1.03493534e-01,-2.50952244e-01,9.45051312e-02]],[[-6.60263658e-01,-4.86506850e-01,-4.62891310e-02],[-6.05915070e-01,6.91424608e-02,2.04617679e-01],[-2.49578401e-01,-3.49210918e-01,-4.36635390e-02]],[[-8.80839899e-02,6.74325153e-02,-5.24185419e-01],[-8.13385844e-01,-1.96483538e-01,-3.37798327e-01],[1.56628832e-01,-1.04290679e-01,-1.10547841e+00]],[[-9.13370848e-02,5.87863088e-01,1.34798735e-01],[-3.53241891e-01,-9.68671367e-02,-1.96324095e-01],[-3.70868206e-01,6.81140542e-01,-6.50694728e-01]],[[3.50826383e-01,1.53906643e-01,7.82651842e-01],[8.20316255e-01,-3.28231305e-02,1.81906432e-01],[3.91745061e-01,-2.67476980e-02,-5.18774807e-01]],[[-3.70399177e-01,-1.29719824e-01,-6.25370085e-01],[-3.90791744e-01,-3.47525984e-01,6.71136454e-02],[1.44031551e-02,-7.14929104e-01,2.79846847e-01]],[[-5.09901308e-02,-7.23770499e-01,-6.51789457e-02],[-1.21784814e-01,5.80710948e-01,2.32989609e-01],[4.40798014e-01,9.26578045e-01,-1.68019742e-01]],[[-5.76337576e-01,4.34774667e-01,-3.81827682e-01],[-1.56465918e-01,-1.68146774e-01,-6.91195875e-02],[-8.02539647e-01,8.08510840e-01,-6.49855793e-01]],[[-4.44022268e-01,2.62656987e-01,-2.53060520e-01],[-2.98914224e-01,1.81861863e-01,-1.60173774e-01],[-4.91004616e-01,-1.06858957e+00,-1.30453455e+00]],[[-4.66033280e-01,5.26788890e-01,3.79807919e-01],[2.31353477e-01,6.87677503e-01,1.14809644e+00],[-3.82051021e-01,-4.53971237e-01,5.80103695e-01]],[[-1.10145032e+00,3.40642691e-01,-4.80475053e-02],[-5.96416891e-01,8.45398605e-02,2.92539865e-01],[-8.53583515e-02,1.71652943e-01,2.63633341e-01]],[[2.13779304e-02,3.46522272e-01,-3.52606662e-02],[3.71254236e-01,-1.85694233e-01,5.24819046e-02],[7.16883957e-01,7.69324973e-02,5.17276168e-01]],[[3.52644473e-01,-6.85091078e-01,-6.75470009e-02],[-5.22005372e-02,-2.25190625e-01,-3.71751994e-01],[7.32682109e-01,-9.34246480e-02,-2.65992224e-01]],[[-4.43173498e-01,6.91861391e-01,-1.43530473e-01],[-2.12997645e-01,-3.54300052e-01,-2.58338898e-01],[1.01946104e+00,1.11254722e-01,8.19934905e-01]],[[6.47736788e-02,2.54048228e-01,6.88758910e-01],[1.78071514e-01,-2.67791539e-01,4.87498105e-01],[-5.43837607e-01,-4.17869270e-01,-8.46624494e-01]],[[5.07352948e-01,7.44605720e-01,-2.07959875e-01],[-1.29266817e-03,6.34065747e-01,2.74575204e-01],[2.41274104e-01,-5.82850933e-01,-3.30276221e-01]],[[4.60493445e-01,-2.85884947e-01,3.73390019e-01],[-1.85703367e-01,-4.88109380e-01,-3.13225627e-01],[-1.28982931e-01,-8.39743763e-02,-1.06877387e+00]],[[3.90753686e-01,1.20887667e-01,3.01162809e-01],[-4.84292775e-01,-3.98414582e-01,4.53949183e-01],[8.69684637e-01,-8.63014340e-01,-5.96886016e-02]],[[4.22125101e-01,-4.22242343e-01,1.75930053e-01],[-4.72727157e-02,3.61427628e-02,-4.86887485e-01],[-7.01397359e-01,-9.16196346e-01,-4.26361322e-01]],[[1.07213998e+00,6.21850081e-02,-2.31006593e-01],[-4.19197917e-01,7.21012568e-03,1.29842266e-01],[4.06831563e-01,2.25064158e-01,-6.48062378e-02]],[[-1.95450395e-01,-2.74801403e-01,-7.51109183e-01],[-8.88909161e-01,1.83564603e-01,1.40400857e-01],[4.85575527e-01,2.50668138e-01,-3.07145238e-01]],[[1.38799146e-01,7.78547347e-01,2.37544831e-02],[1.28546044e-01,3.52725744e-01,5.02840936e-01],[3.06885153e-01,1.72879145e-01,1.07176742e-02]],[[-9.07455742e-01,-7.59963365e-03,-1.34835839e-01],[1.68990627e-01,1.36197180e-01,2.27339212e-02],[3.64039034e-01,-8.28919411e-02,-5.55023253e-01]],[[-4.98845667e-01,-4.88712639e-01,6.59662902e-01],[-3.55465353e-01,-4.83237989e-02,-3.32720786e-01],[1.05802786e+00,3.95196855e-01,1.40702561e-01]],[[-1.23828962e-01,4.31209326e-01,2.51330465e-01],[2.95856982e-01,1.65349275e-01,-1.98154628e-01],[-9.16188836e-01,-1.68871090e-01,4.99628305e-01]],[[-3.09803933e-01,6.49273634e-01,2.86219656e-01],[7.83522487e-01,-1.49403706e-01,-5.78606904e-01],[3.21032286e-01,2.00624257e-01,-3.11970383e-01]],[[1.18647121e-01,9.07822907e-01,1.63959011e-01],[-9.09430757e-02,4.04514670e-01,3.58935177e-01],[-4.82257716e-02,1.24184741e-02,2.74261743e-01]],[[-3.86210352e-01,-2.91127320e-02,1.00706473e-01],[7.39051044e-01,-4.49501067e-01,5.41801937e-02],[-1.32526189e-01,-5.19365907e-01,-5.55116348e-02]],[[7.52174914e-01,-4.13109995e-02,2.00501420e-02],[-7.02769533e-02,-1.59055755e-01,-2.33174235e-01],[-1.08037787e-02,-3.14991102e-02,3.11867595e-01]],[[8.34035337e-01,2.67566472e-01,1.45259768e-01],[4.00652766e-01,1.71763618e-02,3.25723171e-01],[-2.58546591e-01,4.78129297e-01,-4.92342919e-01]],[[4.03353512e-01,-9.58600938e-02,-1.10898471e+00],[-3.47069651e-01,-2.33082235e-01,2.63060749e-01],[-7.33718097e-01,7.36698881e-02,-1.39438018e-01]],[[6.36272132e-02,1.29273459e-02,-3.31695408e-01],[-8.33584785e-01,7.45651543e-01,9.14723948e-02],[-1.58460796e-01,5.23282886e-01,2.63970792e-01]],[[7.48275578e-01,1.73940301e-01,-1.16812857e-03],[2.50716656e-01,4.38937753e-01,4.34762128e-02],[3.09541762e-01,-2.14509919e-01,-3.60990375e-01]],[[1.29450783e-01,1.92406327e-01,-9.34670925e-01],[-9.37398434e-01,-9.56767499e-02,-7.69101501e-01],[-1.05385137e+00,2.63402522e-01,-4.32327330e-01]],[[-1.68382619e-02,4.39604133e-01,-2.40309313e-01],[-1.89761996e-01,-3.50659549e-01,4.78529334e-01],[-1.82967596e-02,9.97113362e-02,2.25400534e-02]],[[-3.56458426e-01,-5.15610874e-01,2.11075395e-01],[5.24667799e-01,1.14723295e-01,-5.61561957e-02],[-4.62018609e-01,-2.94125140e-01,4.34851795e-01]],[[7.95311630e-01,2.63258606e-01,-3.53591949e-01],[-5.82966626e-01,-1.31699651e-01,6.05975628e-01],[4.41522077e-02,-1.97335798e-02,5.62752664e-01]],[[-2.31244951e-01,2.60771483e-01,-1.05651546e+00],[3.59644204e-01,-2.39771962e-01,5.01529463e-02],[7.91416347e-01,-5.91793120e-01,1.73328176e-01]]],[[[-0.974951,-0.9429718,0.28594533],[-0.3982171,-0.36887544,0.06434916],[-0.17547992,0.29797822,-0.17073704]],[[0.5759407,-0.16273133,0.28087792],[-0.32000282,-0.08761064,-0.12643813],[0.6597253,-0.32926372,0.45846787]],[[-0.41537154,1.1213803,-0.6597393],[-0.06350873,0.44666672,-0.9888598],[-0.12622319,0.5306424,0.42668018]],[[0.45132583,0.10452326,-0.2648231],[0.12213095,0.34904236,-0.23062512],[-0.14851564,0.58322,0.82610524]],[[-0.66564053,-0.37552118,-0.25047067],[-0.30184385,0.49592447,-0.5564384],[-0.0985791,0.01223547,-0.0264417]],[[0.0745276,0.09586834,-0.11405519],[-0.06900225,-0.49320552,0.79531264],[-0.41185796,-0.44885635,0.73857737]],[[0.32244223,-0.28134292,-0.33210388],[-0.43891528,0.33917344,-0.58282727],[0.66294265,0.29542965,-0.2880917]],[[0.46187076,-0.36687958,0.34680706],[-0.26091066,-0.98627156,0.14511779],[0.18182556,-0.11013476,-0.4131258]],[[0.14796273,-0.17252363,0.5822392],[0.21719404,-0.635583,0.58020234],[0.5406986,-0.305595,-0.3855386]],[[-0.404518,-0.1598978,0.16810048],[0.09199215,0.4820197,0.36133197],[0.73582035,0.01516086,0.195551]],[[-0.11702839,-0.05115748,-0.34696764],[-0.18390158,-0.03497618,-0.19324893],[-0.16389194,1.0018117,-0.61852854]],[[-0.398981,0.4390565,-0.34012288],[-0.21449521,0.05158109,-0.18359788],[-0.44612613,-0.17645949,0.4508105]],[[0.31012106,0.5528789,-0.18718548],[0.38789326,-0.53341943,0.12707087],[0.09686899,-0.25960463,-0.20916982]],[[-0.14713661,-0.03853185,0.16321243],[-0.7250545,-0.31934226,-0.28358814],[-1.667261,-0.01186466,0.569083]],[[0.02261625,-0.02891496,0.4692326],[0.04024407,-0.15730159,0.24967147],[-0.70238334,-0.82064295,0.02675078]],[[0.52081805,-0.08396791,0.17449889],[-0.21151201,0.04454849,0.14180975],[-0.19605397,-0.6772821,-0.15889113]],[[-0.11013131,0.3643093,-0.07946215],[1.155645,-0.1104364,-0.0270114],[0.79784673,-0.14773837,0.19679706]],[[0.28656203,0.07041344,0.01824608],[-0.42345318,-0.07557207,0.19837786],[-0.103535,0.2352845,-0.8244554]],[[-0.10282145,0.14567354,-0.5279377],[-0.00664559,0.43067008,-0.17616247],[0.5363193,-0.46788058,0.5815935]],[[-0.00950329,0.11142746,-0.08188142],[0.23691784,0.99899673,0.5587315],[0.22326289,0.46658707,0.32270387]],[[0.02072705,-0.16414538,-0.09260817],[-0.22218928,-0.10630365,0.33861467],[-0.21514177,0.08814212,-0.6062485]],[[0.73551494,0.28941974,0.17721808],[0.30019575,-0.29215118,0.00650032],[0.27630395,0.01503868,0.05036641]],[[0.56309813,0.11299681,2.1382308],[-0.12482822,0.41374943,0.9262589],[-0.3551166,0.40088528,0.3544567]],[[0.90383196,-0.5755945,0.0350013],[0.36228603,-0.11823405,-0.23480985],[-0.7597486,-0.26783398,0.24344534]],[[0.6624161,-0.27028617,0.43353167],[0.592177,0.15442863,0.8400139],[1.3696775,0.38640547,0.7580638]],[[-0.46609688,0.08701941,-0.37153792],[-0.07492732,0.4015427,-0.21941723],[0.116413,-0.84206754,-0.74756426]],[[0.12435113,-0.1263717,0.93845844],[0.31750622,0.5763859,0.8214035],[0.14028764,-0.40982494,0.06682912]],[[0.21239555,-0.246902,-0.35583225],[-0.14928791,-0.7233412,0.18605529],[0.14439628,-0.13672757,0.2596412]],[[-0.514526,-0.19859086,-1.5877692],[0.47560975,-0.14643858,-0.4935734],[0.5006851,-0.34118715,-0.60437715]],[[-0.7789128,0.60231876,0.14516376],[-0.7747036,0.27492276,-0.2992567],[0.08385032,-0.03126164,-0.05635968]],[[-0.57522357,-0.20236008,-0.01255798],[-0.00252703,-0.17367151,-0.15216383],[-0.29177886,-0.2171825,-0.21813276]],[[-0.16027814,0.47708473,0.02991137],[0.20260336,-0.01207945,-0.4422201],[-0.21873653,0.60390097,-0.3042556]],[[0.3774861,0.11359306,-0.34965417],[0.08152427,-0.01459128,-0.01541603],[-0.03794378,-0.21747668,0.9777236]],[[0.93335766,0.50477743,-0.0208243],[0.83726615,0.00966417,0.3833291],[-0.03931426,-0.04237289,-0.6498152]],[[-0.38082635,0.59754294,-0.23208305],[-0.27318335,1.0336226,-0.23359817],[0.29112685,-0.4190103,-1.2112039]],[[-0.10707732,0.12209965,-0.5592587],[0.75913656,0.342683,-0.6378875],[-0.4389121,-0.33461267,-0.10833615]],[[-0.15408216,0.5121404,-0.52380216],[0.35882127,0.49456576,-0.5310717],[0.14486608,0.49939752,0.04465765]],[[0.26265505,-0.00410588,0.15070759],[0.32113466,0.1989407,0.09571822],[0.5515552,-0.26454556,0.12018009]],[[-0.6671835,-0.50277424,0.18016668],[-0.5492508,-0.22471747,0.6223166],[-0.04402965,-0.06989398,-0.05681919]],[[0.6087193,-0.07876011,0.0779059],[0.9006098,0.18246362,0.3286804],[-0.09725291,0.57650965,0.04724294]]],[[[-0.08338518,0.6223929,0.5039189],[-0.23490945,0.02076342,0.30796272],[-0.2666113,0.12507646,-0.42666996]],[[0.26523533,-0.08327918,0.44774452],[-0.5141794,-0.50665915,-0.21253254],[-0.56307197,0.2168776,0.23345718]],[[0.19697833,0.7183343,-0.15665923],[0.34591955,0.22156024,0.51025766],[0.17232665,-0.51891476,-0.46177456]],[[-0.11015355,-0.39995155,-0.17354609],[0.7942973,0.31707636,-0.9054351],[0.02791475,-0.50265336,-0.30591053]],[[-0.8774725,0.27695298,-0.47437605],[0.4094812,-0.2532261,-0.9747551],[0.33787078,0.53770447,-0.07952035]],[[-0.75550693,-0.27365947,0.17207305],[-0.66346693,0.11822388,0.8920496],[-0.14437678,0.24101497,-0.3152048]],[[0.76028556,-0.20600723,-0.42554694],[0.9563754,0.23989016,-0.5920485],[-0.09999309,-0.38670403,-0.55281097]],[[-0.7323103,-0.46966386,0.4202132],[0.23646174,-0.598073,0.44119817],[-0.8245744,-0.28653735,-0.21917522]],[[-0.299755,0.43446878,0.06377778],[0.53295416,-0.43244857,0.6576835],[-0.27023295,-0.18930598,0.0615244]],[[0.578719,-0.71042216,0.6881865],[0.01041637,-0.40894866,0.27496558],[-0.30550075,-0.69560397,0.24092552]],[[0.41594097,1.047699,0.02730961],[-0.00818662,-0.25527728,-0.31062937],[0.46323732,-0.28577125,0.08195429]],[[-0.16631185,0.40475383,-0.08348184],[-0.29354858,0.28036922,0.02006133],[-0.3883964,-0.10869059,-0.5724284]],[[-0.3300384,-0.33637637,-0.4537142],[0.46069974,0.69381744,-0.20409766],[1.1853597,-0.22840312,0.04617158]],[[-0.11987352,-0.48046213,-0.0149792],[-0.12124701,0.2703597,0.25883147],[-0.36949995,-0.12873173,-0.13994968]],[[-0.11052148,1.0253584,1.102898],[-0.96926117,-0.00856147,-0.18457666],[0.7042338,0.04373269,0.46041253]],[[-0.09766845,-0.60648936,-0.08143754],[0.18312232,0.70797473,0.41802123],[-0.33534065,-0.04593854,-0.7175526]],[[-0.26399407,0.29244918,0.4556943],[0.2969373,0.33808714,-0.25092298],[0.911942,-0.34026572,-0.5635637]],[[-0.10959504,0.40003148,0.7625152],[0.1602429,-0.6352739,-0.15927875],[-0.71780896,0.12461255,-0.13909787]],[[-0.18360715,-0.34173337,-0.09351668],[0.4557418,-0.05318384,0.11713719],[0.347998,-1.0924491,0.25811854]],[[0.18249992,0.5353786,-0.14374007],[0.12104946,-0.31670037,-0.5958738],[0.33736998,-0.12098992,0.5344854]],[[-0.24579865,-0.27614132,0.23493916],[0.36727226,-0.673671,0.04299723],[0.34073848,-0.02542932,-0.01750386]],[[-0.4400315,0.28789997,0.22780003],[-0.40272677,0.3854326,0.7549995],[-0.52115715,-0.17729038,-0.17354229]],[[0.6239633,-0.04400776,0.00688331],[-0.3384056,0.49681252,0.40386885],[0.1464829,-0.01475969,0.34076682]],[[0.90480834,-0.0821438,-0.9410614],[0.24940187,0.2761969,0.19061036],[-0.11653068,-0.33972347,-0.05937931]],[[0.42120132,-0.07931321,-0.31207567],[0.01230947,-0.45321897,-0.33793843],[0.2011645,0.07489192,0.05142519]],[[-1.1405729,0.36726144,-0.16600433],[0.01904711,0.27166328,-0.00236435],[-0.25872457,0.00697578,0.40324968]],[[-0.4338678,-0.32012156,-0.71248025],[-0.29761693,-0.6823927,0.14357159],[0.8217676,0.10765673,-0.6956]],[[-0.62521124,0.16207151,0.12568393],[0.1789125,0.21007693,-0.12854767],[0.01170929,-0.67660517,-0.3237576]],[[-0.34948623,0.31609222,-0.6632473],[0.39818764,0.03718876,-0.69123435],[-0.19818375,0.10533395,0.0122049]],[[-1.2076287,-0.5218628,1.1025857],[-0.6774548,-0.1937933,0.24217822],[-0.09725391,0.18164088,-0.13580035]],[[0.5470766,-0.24481001,-0.38783598],[0.05589109,0.11470275,-0.4530323],[1.6686437,-0.42896074,-0.7320266]],[[0.64426917,-0.07654782,-0.5548406],[0.05976989,0.6896053,0.15614466],[0.18639083,-0.5756209,-1.157698]],[[1.2416506,-0.20132075,-0.6778201],[-0.50713754,-0.8796605,0.1200508],[0.95964664,0.26088956,0.01155998]],[[-0.63212353,-0.2153596,-0.10316426],[-0.19101524,0.2772122,-0.05400464],[-0.10276312,0.37259415,0.3284438]],[[-0.69433343,-0.29729933,0.1883275],[0.27493474,-0.5339584,-0.20733933],[0.16122794,-0.7294144,0.17437534]],[[-0.41151306,-0.07626686,0.3569892],[-0.45103967,-0.42171946,-0.39879563],[0.12506157,-0.6302514,-0.53757465]],[[0.42166013,0.00715856,0.50339085],[0.15490364,0.1593173,0.5809548],[0.45105168,-0.54698604,0.02809527]],[[-0.14243676,0.00869741,-0.02437925],[-0.51309747,0.7414182,-0.20531623],[0.7896537,-0.31490988,0.72607005]],[[-0.3258924,-0.3280404,-0.06841253],[-0.02950802,-0.36901245,-0.50108874],[-0.3959432,-0.14585395,0.08544362]],[[-0.35096213,-0.38176876,0.3047566],[-0.7994255,0.48746294,-0.5181953],[1.1787413,-0.80006176,-0.22508346]]],[[[4.93574917e-01,5.73927402e-01,5.81162274e-01],[-1.69379994e-01,2.75870591e-01,-4.92919117e-01],[-5.17633617e-01,-3.04604292e-01,-6.76932186e-02]],[[-3.95101726e-01,3.67170960e-01,7.39398450e-02],[-8.43562007e-01,-1.26509607e-01,5.38899302e-01],[2.48624578e-01,-5.90121567e-01,1.09573796e-01]],[[1.03549786e-01,3.71714830e-01,8.58302116e-01],[-5.17660379e-01,5.46021938e-01,-1.87068852e-03],[8.59668404e-02,-1.05640858e-01,-2.77413335e-02]],[[-7.05249757e-02,-1.52678922e-01,-1.58113346e-01],[-8.17006454e-02,4.81819570e-01,7.28221953e-01],[6.18369579e-01,8.98167565e-02,-8.14630330e-01]],[[2.83294141e-01,4.14280593e-01,-1.43799365e-01],[2.26255476e-01,-4.21718538e-01,5.80012858e-01],[-1.45919308e-01,5.72341800e-01,-2.10278347e-01]],[[4.64839607e-01,-1.83699355e-01,4.42362309e-01],[-8.24692398e-02,6.40214682e-01,2.26452947e-01],[4.95482028e-01,3.58319044e-01,-2.10288954e+00]],[[-7.13661313e-02,-9.98586044e-02,-3.17855835e-01],[-4.44881260e-01,-3.67678642e-01,6.63224757e-02],[-5.19859195e-01,5.55085003e-01,-4.00077730e-01]],[[3.61622125e-01,5.46788037e-01,-5.32142743e-02],[-1.89037129e-01,-8.33068609e-01,-2.61988401e-01],[-8.27115029e-02,-3.11993659e-01,-1.02819078e-01]],[[-2.82150935e-02,-1.27442911e-01,8.00262511e-01],[3.50004941e-01,1.59819588e-01,-5.08572422e-02],[5.28395116e-01,6.93289816e-01,-3.51765305e-01]],[[-3.17360163e-01,5.25418162e-01,-6.97342634e-01],[-8.54901433e-01,-1.25752911e-01,-7.84428343e-02],[2.46428370e-01,9.42921758e-01,4.10248712e-02]],[[5.12361348e-01,1.16186619e-01,2.75102377e-01],[4.28621143e-01,-3.47586960e-01,-7.13320076e-01],[3.55320215e-01,-4.40560341e-01,-1.05193205e-01]],[[4.33178134e-02,1.74768612e-01,7.28875339e-01],[-6.70672953e-02,-3.33895177e-01,4.51842785e-01],[-1.87946513e-01,-2.48414531e-01,-5.09834886e-01]],[[3.42583209e-01,-2.73095518e-01,2.88002700e-01],[-4.64619517e-01,-3.54702562e-01,2.00607833e-02],[-1.57295242e-01,-1.29318237e+00,-5.72875321e-01]],[[8.77644345e-02,4.49765846e-02,1.54300228e-01],[6.77126646e-02,3.05944860e-01,1.55592814e-01],[-1.81561217e-01,-4.33990285e-02,4.44073349e-01]],[[4.17737573e-01,-4.23894912e-01,1.13327533e-01],[-1.57497719e-01,-1.89698264e-01,2.42135406e-01],[-3.54345441e-01,-4.85613793e-01,1.56259227e+00]],[[4.79394078e-01,2.41560116e-02,-6.20836057e-02],[4.10585821e-01,3.51944625e-01,-5.67451179e-01],[1.10267782e+00,5.12348175e-01,-5.57178855e-01]],[[-9.00055766e-02,3.18780094e-01,-1.80826679e-01],[-6.82911634e-01,7.02512497e-03,2.23264024e-01],[2.17623487e-02,-6.08059987e-02,-5.42916417e-01]],[[-2.30379179e-01,-2.88299978e-01,7.22434558e-03],[8.66632104e-01,-5.60673960e-02,5.42380393e-01],[1.47617564e-01,5.16813993e-01,-4.40452069e-01]],[[-1.06013432e-01,2.11392447e-01,-2.83501357e-01],[-4.28073764e-01,4.36254889e-02,-1.53015390e-01],[4.62140977e-01,3.23766738e-01,-3.53049159e-01]],[[-1.00154318e-01,-1.54075801e-01,-4.89668518e-01],[2.21311644e-01,3.81408662e-01,-1.88499153e-01],[4.18340266e-01,-3.44506025e-01,-2.67737389e-01]],[[-4.36418355e-01,-2.38039747e-01,7.86931992e-01],[-1.15748234e-01,-1.05858065e-01,4.06564116e-01],[-3.48061502e-01,-2.88155645e-01,-2.00671420e-01]],[[-1.72788277e-01,8.67834538e-02,-1.79119945e-01],[-2.12937534e-01,3.88346374e-01,-3.07646573e-01],[8.72859061e-01,-1.22732103e+00,4.36335683e-01]],[[-2.85354018e-01,2.23311588e-01,-2.78959632e-01],[2.25307867e-01,1.10353366e-01,1.57145157e-01],[-6.71112835e-01,-9.61371660e-01,5.32911003e-01]],[[5.67676723e-01,-1.85601071e-01,1.49415180e-01],[-6.88858181e-02,1.92959368e-01,-5.21560609e-01],[-2.85454094e-01,-3.76310945e-01,5.23936272e-01]],[[-1.07946910e-01,-4.33970630e-01,-5.20667672e-01],[-2.48382956e-01,-2.99011618e-01,-1.14788515e-02],[4.74889815e-01,5.90167820e-01,2.09035993e-01]],[[1.86431929e-02,2.06326872e-01,-6.17603838e-01],[-2.51015306e-01,2.51673818e-01,1.45915225e-01],[6.52056694e-01,1.11339164e+00,2.17784151e-01]],[[2.41266325e-01,2.37872288e-01,-8.41199979e-02],[-7.81316221e-01,6.40541688e-02,-3.09998930e-01],[1.30131948e+00,3.63097847e-01,-7.29589641e-01]],[[-2.82652110e-01,3.20327044e-01,-4.98534828e-01],[-5.20766914e-01,1.65487811e-01,1.91516697e-01],[1.48530185e+00,5.97163849e-02,6.22197688e-01]],[[1.41462713e-01,2.15419963e-01,4.96826321e-01],[-3.07825059e-01,-5.40933274e-02,-1.67681769e-01],[2.20981259e-02,-4.25790846e-01,1.57632411e-01]],[[6.12813830e-01,-1.42545536e-01,-1.85515732e-01],[5.21850467e-01,3.92253369e-01,-6.06807590e-01],[-1.73190683e-01,-1.77197605e-01,2.58255571e-01]],[[1.09363794e-01,-4.08775583e-02,-2.62983829e-01],[-1.17960773e-01,8.57892558e-02,2.86965836e-02],[-1.95993125e-01,4.83128548e-01,5.60521707e-02]],[[1.81278944e-01,4.93401885e-01,-7.16454029e-01],[7.01443255e-01,4.17170316e-01,-1.81891903e-01],[-1.27178319e-02,-2.00638354e-01,2.85660267e-01]],[[-1.84973940e-01,3.65081802e-02,4.60346490e-01],[-3.37864101e-01,6.65292203e-01,1.70619860e-01],[2.62511104e-01,-2.77593523e-01,-2.83507138e-01]],[[-9.27719712e-01,1.35615051e-01,-4.36337106e-02],[7.16784298e-02,-1.75658733e-01,-6.68070436e-01],[-9.32199001e-01,-4.01264399e-01,-8.39340746e-01]],[[-5.67554124e-03,-7.63599396e-01,5.09735882e-01],[1.79636795e-02,-6.29738092e-01,1.37511194e-01],[-3.14279735e-01,7.40988135e-01,-4.75881547e-01]],[[2.90032268e-01,4.96020168e-01,1.21874690e-01],[6.61780834e-01,-6.71286404e-01,1.68795377e-01],[-8.89819562e-01,2.24998713e-01,-3.50038141e-01]],[[5.19644856e-01,-5.14152527e-01,1.82392392e-02],[3.28677088e-01,7.63841391e-01,8.99779350e-02],[-6.10189736e-01,2.35088244e-01,-3.30693603e-01]],[[-3.34715098e-01,-8.89223516e-01,-2.98459738e-01],[-1.20232001e-01,-2.18668595e-01,-6.06320024e-01],[-3.50522012e-01,7.14572012e-01,-6.87127590e-01]],[[1.01246583e+00,1.88562423e-01,-1.66155368e-01],[-4.94206958e-02,-1.32438824e-01,1.76888674e-01],[5.63128963e-02,3.64304423e-01,-2.75603205e-01]],[[-3.08213592e-01,1.18267410e-01,4.36173290e-01],[-2.83240136e-02,-1.39020073e+00,6.72554135e-01],[-5.61562419e-01,-5.23753941e-01,5.36732785e-02]]],[[[-1.23341858e-01,-3.48958939e-01,3.25592697e-01],[-3.42788659e-02,-5.32859087e-01,2.14059576e-01],[-6.59292877e-01,-1.50359124e-01,2.33725294e-01]],[[-2.17023045e-01,-1.77164599e-01,-7.54653096e-01],[8.33458379e-02,6.35779509e-03,-3.65450680e-01],[1.06300199e+00,8.18960220e-02,-1.90072566e-01]],[[-5.51223218e-01,-1.14795968e-01,-7.82415390e-01],[-2.69745529e-01,3.34020466e-01,-2.65197009e-01],[1.55731440e-01,1.08145915e-01,3.38988900e-01]],[[2.93873996e-01,-1.50793254e-01,1.33318484e+00],[1.92843545e-02,9.43930268e-01,6.21538401e-01],[-1.26580164e-01,-1.93808317e-01,4.68258075e-02]],[[-3.48115514e-04,9.92294550e-02,-1.50371805e-01],[-7.47019887e-01,4.57344890e-01,-4.87074256e-01],[-6.25311350e-03,2.05258593e-01,-6.74028099e-01]],[[4.00556862e-01,-1.48300901e-01,1.60537511e-01],[5.42804182e-01,5.57544112e-01,-1.54353574e-01],[-1.38502672e-01,2.96606898e-01,-1.29603878e-01]],[[-5.55720747e-01,4.83617276e-01,4.48041022e-01],[4.27506506e-01,2.67135412e-01,-1.34613905e-02],[1.66869015e-01,-3.05949360e-01,3.00663799e-01]],[[-8.74283388e-02,2.70625323e-01,1.54732019e-01],[3.25155944e-01,-1.67575106e-01,-7.73556650e-01],[-6.64540231e-01,4.59740818e-01,7.97025263e-02]],[[7.33468175e-01,2.78446347e-01,6.75314605e-01],[9.99866903e-01,5.67768097e-01,2.24691585e-01],[5.97957015e-01,1.52805254e-01,4.34094846e-01]],[[7.33342230e-01,2.80784011e-01,1.34377703e-01],[6.50157452e-01,8.33512619e-02,3.02887410e-01],[7.86938310e-01,2.71111280e-01,-5.48465252e-01]],[[2.88717926e-01,1.59790844e-01,1.57299973e-02],[3.07150126e-01,-1.10947611e-02,5.88111639e-01],[-1.64778784e-01,-2.63793111e-01,-1.35280281e-01]],[[1.34012789e-01,2.15484232e-01,-4.78700012e-01],[-7.27513880e-02,-4.98163730e-01,1.68004096e-01],[-7.05286980e-01,3.00410330e-01,2.15783179e-01]],[[5.71811609e-02,-5.98505020e-01,-2.65611231e-01],[4.61925000e-01,-5.89073002e-01,3.15303296e-01],[1.03481136e-01,-3.37977678e-01,-3.71051073e-01]],[[3.83382857e-01,5.65007515e-02,1.95332780e-01],[-7.07141683e-02,3.56644019e-02,5.36622226e-01],[-7.90567398e-01,1.72132313e-01,3.20426039e-02]],[[3.14201146e-01,4.21423241e-02,4.20946348e-03],[1.74719900e-01,-7.92090595e-02,3.92131895e-01],[5.04772663e-01,4.06435579e-01,-3.47131252e-01]],[[4.40331459e-01,2.61098206e-01,6.46053553e-01],[5.00002861e-01,6.05104677e-02,3.40240538e-01],[-2.25450322e-02,6.12838566e-01,-4.44338530e-01]],[[4.34437305e-01,2.11820409e-01,-6.56437397e-01],[2.52206981e-01,8.09258819e-01,-3.57212931e-01],[1.14133859e+00,3.41726422e-01,8.13264027e-03]],[[-1.42693236e-01,7.26785809e-02,-3.05093735e-01],[-4.06615943e-01,-3.08821164e-02,4.81843799e-01],[-5.92837594e-02,1.68787852e-01,-3.51252496e-01]],[[1.47300020e-01,6.51797116e-01,-1.35567442e-01],[-2.04960957e-01,1.25890955e-01,7.03318238e-01],[3.04971486e-01,-2.91826338e-01,3.79961878e-01]],[[2.78388083e-01,4.67614442e-01,2.98440993e-01],[4.29274701e-02,5.50902247e-01,6.22486293e-01],[7.91894019e-01,3.82209599e-01,2.21187925e+00]],[[-3.32992911e-01,-3.81330371e-01,-1.39506674e+00],[2.94615209e-01,-1.57028824e-01,-1.09226704e+00],[1.65667236e-01,-2.63524354e-01,-2.20330492e-01]],[[3.68194729e-01,8.63592550e-02,5.69746554e-01],[6.74015462e-01,5.46068907e-01,3.99436623e-01],[-7.43714124e-02,-6.05515130e-02,-4.20293920e-02]],[[4.39006984e-01,-3.10791969e-01,1.73059213e+00],[6.11706786e-02,-2.91417986e-01,9.79277015e-01],[-1.13478974e-01,-7.01031536e-02,4.02560800e-01]],[[1.11217469e-01,5.08304060e-01,3.69921565e-01],[4.46404576e-01,1.02733588e+00,-3.61857414e-01],[3.44235271e-01,-6.60039410e-02,8.64546299e-01]],[[8.22842717e-01,4.23570812e-01,7.11051702e-01],[1.79414779e-01,-4.05906625e-02,4.85642850e-01],[2.66309112e-01,8.54135096e-01,1.11217344e+00]],[[-2.86213338e-01,4.70918417e-01,-3.82726341e-02],[-4.72315460e-01,2.89996028e-01,-1.54046595e-01],[-7.71935344e-01,3.90727073e-01,-6.98357165e-01]],[[2.03168043e-03,3.01832467e-01,9.69667256e-01],[2.74602771e-01,-1.76853910e-01,7.89672613e-01],[-3.22060525e-01,-3.38279456e-01,6.82613611e-01]],[[1.37607634e-01,5.54227158e-02,5.01084864e-01],[5.07528841e-01,1.32080480e-01,3.47874105e-01],[-2.88001686e-01,-2.08882987e-02,-2.08083540e-02]],[[-9.28954720e-01,1.07830621e-01,-1.13469672e+00],[-2.62400538e-01,2.84380019e-01,-4.62624818e-01],[-1.01081394e-01,5.63883960e-01,-4.49878037e-01]],[[-3.76156718e-01,-4.95142460e-01,-2.23460104e-02],[2.02845857e-01,3.76902074e-02,3.10731858e-01],[-6.59797311e-01,-4.39651340e-01,-5.96996665e-01]],[[-8.21405232e-01,3.78213972e-01,-8.34186852e-01],[-1.15773320e-01,2.48933822e-01,1.76256150e-02],[-9.68260765e-02,4.78459299e-01,1.21206574e-01]],[[5.47656640e-02,3.20250332e-01,-3.11438382e-01],[-4.62675214e-01,4.30872202e-01,-1.42595723e-01],[1.43803060e-01,4.80701149e-01,3.50705087e-01]],[[-8.55153520e-03,-2.01933637e-01,1.77207470e-01],[1.68666303e-01,1.84950039e-01,2.84023464e-01],[6.49151921e-01,5.30585051e-01,1.31065696e-01]],[[-2.75203437e-01,6.85438275e-01,-5.97052813e-01],[-6.27532244e-01,-1.86582789e-01,-7.26584375e-01],[-1.16141453e-01,3.26546431e-01,-5.89362919e-01]],[[2.70533673e-02,1.02701746e-01,-2.14310199e-01],[-2.24649981e-01,-1.19889416e-01,9.02581289e-02],[7.67409056e-02,-2.53910571e-02,-5.95479012e-01]],[[-3.98896247e-01,3.33656669e-01,-4.90425587e-01],[-7.96652853e-01,3.72041851e-01,-1.18668169e-01],[-1.92681000e-01,1.13599934e-01,2.82751977e-01]],[[4.87566948e-01,5.46899199e-01,-5.53455412e-01],[-4.54845250e-01,1.88517705e-01,-2.31390789e-01],[2.10658103e-01,-4.21101116e-02,-5.01676321e-01]],[[3.41699809e-01,7.49156401e-02,7.16106892e-01],[4.44573134e-01,1.41413122e-01,5.73960245e-01],[3.67489785e-01,6.49224460e-01,3.39068502e-01]],[[-7.04280019e-01,1.22795694e-01,4.54884410e-01],[3.95869762e-01,2.10884050e-01,2.92034477e-01],[-3.73961896e-01,-1.68710798e-02,5.34585714e-01]],[[6.73560202e-01,-7.35239983e-01,2.83034146e-01],[4.34912890e-01,4.19664860e-01,1.00520039e-02],[8.51020038e-01,-1.75281793e-01,1.19981140e-01]]],[[[-3.18105668e-02,-4.65245545e-01,-5.48721194e-01],[5.95589101e-01,2.84179509e-01,-5.60059845e-01],[1.05871761e+00,-1.15841262e-01,3.66342515e-01]],[[-8.67275834e-01,-1.28159836e-01,-5.28196752e-01],[-1.63077548e-01,3.48303199e-01,-6.63369775e-01],[-2.68936336e-01,-2.97035635e-01,8.64178017e-02]],[[-4.58662033e-01,-6.29861593e-01,2.71828115e-01],[5.41649997e-01,5.73193207e-02,-5.26043713e-01],[4.00254913e-02,-1.61910225e-02,-4.98878164e-03]],[[-1.55041412e-01,-8.10336590e-01,4.49572615e-02],[-3.29251051e-01,-2.03187138e-01,-3.38413090e-01],[-2.38728710e-03,1.01732902e-01,-3.29572886e-01]],[[-2.41979897e-01,-2.90112436e-01,2.18971580e-01],[-2.95832485e-01,-4.51894492e-01,-1.17565118e-01],[4.62522775e-01,-6.78192019e-01,1.40819838e-03]],[[2.21457809e-01,4.22007680e-01,-2.88286120e-01],[-1.71693742e-01,3.71891916e-01,-2.24197075e-01],[4.30582166e-01,-1.28934824e+00,7.05402553e-01]],[[4.53551590e-01,6.31927669e-01,6.95614517e-01],[-5.82530200e-01,1.62318945e-01,2.37535372e-01],[7.82744646e-01,-9.49265361e-01,8.33094597e-01]],[[2.10285857e-01,2.86774546e-01,9.39942300e-02],[-4.27072972e-01,-3.91837060e-02,4.55311835e-01],[2.17007607e-01,-9.66430604e-02,8.58886123e-01]],[[-1.68475553e-01,-1.17524460e-01,1.40797034e-01],[1.45630790e-02,-2.08005771e-01,2.23387420e-01],[5.24765074e-01,-7.32327163e-01,8.46991777e-01]],[[-2.66512364e-01,-1.34479493e-01,7.96407089e-02],[-1.71792775e-01,-3.50938767e-01,-1.01318741e+00],[8.32014322e-01,5.57239473e-01,5.90938926e-01]],[[3.65081429e-02,-7.05170259e-02,-1.24584241e-02],[6.79817617e-01,1.40199155e-01,4.57045257e-01],[8.49090517e-01,1.63610458e-01,4.11770672e-01]],[[-2.46163517e-01,-1.27141610e-01,3.29582423e-01],[2.08333209e-01,-2.46839389e-01,-2.23204330e-01],[-1.15780675e+00,-2.25912873e-02,-2.75901884e-01]],[[1.72993258e-01,3.31411600e-01,4.49783236e-01],[1.24918632e-02,1.14440784e-01,2.43371222e-02],[4.04540926e-01,1.34706572e-01,7.26168394e-01]],[[-2.42187560e-01,-2.76942819e-01,-5.10011792e-01],[1.00031629e-01,1.85288072e-01,-6.45885110e-01],[5.55074513e-01,1.64035797e-01,1.54431835e-01]],[[-2.96108365e-01,-4.82548088e-01,-6.09034449e-02],[-8.64257738e-02,1.17720403e-01,-2.52807349e-01],[1.39791248e-02,3.47708732e-01,-4.94687119e-03]],[[4.98201758e-01,-1.82838142e-01,2.99604625e-01],[-3.05346578e-01,2.24080920e-01,-4.19385552e-01],[6.83253884e-01,1.23451933e-01,-3.45823824e-01]],[[1.81933016e-01,8.73974562e-02,1.10284865e-01],[6.99060798e-01,-1.13215797e-01,3.10959876e-01],[-8.46949965e-02,1.30807251e-01,9.50731516e-01]],[[8.87390599e-02,-1.53949246e-01,1.86584920e-01],[7.90756047e-01,3.97315979e-01,6.94083810e-01],[-5.23992598e-01,5.63378990e-01,1.48585474e+00]],[[4.03728426e-01,6.08330891e-02,-2.63323672e-02],[-2.36573890e-01,-3.19477618e-01,-6.46195948e-01],[-5.70060253e-01,2.62195289e-01,-2.87162870e-01]],[[6.27644002e-01,-3.20023686e-01,3.05125892e-01],[3.16351593e-01,-7.03713119e-01,6.80800140e-01],[-1.85815051e-01,9.66572404e-01,4.39738214e-01]],[[5.72200835e-01,-4.04668927e-01,2.25059420e-01],[3.85986179e-01,-5.90609491e-01,-2.61631101e-01],[-2.11814806e-01,-1.60167560e-01,1.00833446e-01]],[[1.11118627e+00,3.06356788e-01,5.05801886e-02],[4.12770808e-02,1.62996024e-01,5.17938659e-02],[6.56000301e-02,2.07450658e-01,-3.25114340e-01]],[[-4.88392532e-01,3.60957175e-01,1.97136149e-01],[-1.79496631e-01,1.34180024e-01,4.07349974e-01],[-9.35619414e-01,-6.21233225e-01,-1.07636176e-01]],[[3.12560827e-01,3.84033173e-01,5.05332984e-02],[2.00577989e-01,2.00418219e-01,3.29867631e-01],[2.38766998e-01,-6.97592556e-01,-4.09285009e-01]],[[-5.73220134e-01,-1.15301333e-01,-1.16845869e-01],[-1.34962454e-01,-4.45846945e-01,9.23900027e-03],[1.64397374e-01,8.58705118e-02,2.77122378e-01]],[[-5.00157587e-02,1.54637173e-01,-3.77547383e-01],[6.01860811e-04,1.79462358e-01,-4.71069455e-01],[2.69365102e-01,1.06018329e+00,1.24801266e+00]],[[-3.43830824e-01,-3.65137041e-01,-1.02800541e-01],[6.77992404e-01,-7.42401838e-01,-1.89541802e-01],[-1.59086078e-01,-3.08823943e-01,2.38940373e-01]],[[7.77237909e-03,5.93555272e-01,3.79295833e-02],[1.83800399e-01,-7.76110351e-01,-2.29385778e-01],[1.32904661e+00,6.76448762e-01,1.50642645e+00]],[[1.68469682e-01,6.25519231e-02,4.89404708e-01],[3.30876589e-01,-2.06930056e-01,-1.86373278e-01],[3.12720001e-01,-3.09561074e-01,4.02465016e-02]],[[1.97765678e-01,-2.77959853e-02,-2.00060591e-01],[1.65825617e-02,2.18271911e-01,1.54106498e-01],[3.27118874e-01,-4.71270978e-01,1.24843121e-01]],[[3.42939049e-01,8.03246498e-02,-7.29793161e-02],[-2.47429475e-01,-7.57687315e-02,-8.21181759e-03],[4.27415639e-01,-7.06692219e-01,2.04893008e-01]],[[2.94248946e-03,-5.73575497e-01,3.51225197e-01],[-9.26776767e-01,-1.04282963e+00,-9.73679245e-01],[-1.19418576e-01,-2.39026323e-01,1.10746109e+00]],[[1.51657209e-01,3.83127332e-01,4.63361368e-02],[1.39130708e-02,5.13236344e-01,5.72729170e-01],[-1.61831155e-02,-2.51467824e-01,5.07071316e-01]],[[2.13012025e-01,-2.59585977e-01,1.48295119e-01],[-4.92996462e-02,-5.62041819e-01,-7.60573924e-01],[3.33953500e-01,-1.34324634e+00,-1.38658658e-01]],[[-5.55177391e-01,2.57551849e-01,5.65432787e-01],[6.42679393e-01,2.03474626e-01,-4.71040636e-01],[-2.12575555e-01,-4.72249510e-03,-6.49201572e-02]],[[3.36664915e-01,1.69686094e-01,-2.41724476e-01],[2.53357053e-01,-7.09083736e-01,-1.44554049e-01],[1.35928979e-02,-5.68301439e-01,7.49812424e-01]],[[-1.77465290e-01,1.83441993e-02,8.97098854e-02],[5.11666834e-01,3.07470679e-01,-4.35677499e-01],[-1.44627899e-01,-4.59817320e-01,-3.15300107e-01]],[[7.10902452e-01,-2.66881976e-02,4.58486199e-01],[1.74928501e-01,3.82351488e-01,-1.82490245e-01],[3.89695317e-01,3.16777468e-01,-1.39276087e-01]],[[-8.06897998e-01,-8.63964558e-02,-2.90862769e-01],[-4.98767197e-01,-5.25137961e-01,6.27240598e-01],[-5.52940011e-01,-6.65683925e-01,-6.95083141e-01]],[[1.17181823e-01,-3.06881964e-01,3.97253424e-01],[-1.18130341e-01,8.36457089e-02,-3.14565718e-01],[-5.51376045e-01,-3.28875870e-01,-3.29273432e-01]]],[[[0.8139622,0.10374591,-0.22159442],[-0.58208066,-0.20715557,0.53866404],[-0.36108118,0.15702252,-0.10386423]],[[-0.19285646,-0.4981233,0.59818196],[0.17497961,-0.43822747,0.34578395],[0.05782263,-0.23829648,0.45245704]],[[0.47327507,0.17961751,0.316716],[0.3357041,0.13679126,-0.10972209],[0.19105884,-0.01867123,-0.20565398]],[[-0.8185037,-0.3964921,0.1700968],[0.481311,-0.75943017,0.81729484],[0.3494688,0.37519196,0.81236523]],[[-0.7976805,0.04452168,0.11231124],[-0.33928674,-0.17559588,-0.12092149],[0.558762,0.3471964,0.12990472]],[[0.017732,-0.11472667,0.24223843],[-0.25397435,-0.60210365,-0.43870488],[-0.82773006,-0.09811743,-0.78603923]],[[0.37432012,0.2610561,0.21174996],[-0.11981764,-0.3905642,-0.5291121],[-0.45182273,-0.19031137,0.11081593]],[[0.7429956,-0.1035011,-0.30918708],[-0.5000547,-0.42571363,0.2537425],[0.00685385,-0.5827867,0.5833783]],[[0.28267297,-0.4624409,-0.11307614],[0.20216869,0.12267286,-0.67842335],[-0.37367505,0.51504034,0.15656395]],[[0.05141502,-0.32468235,0.11270498],[-0.59830713,0.37909937,0.5954045],[-0.08010305,0.11455249,0.25086108]],[[0.02969924,-0.3471398,0.06585735],[0.3221918,-0.66361773,-0.33464637],[1.0414103,0.3638444,0.5381765]],[[-0.01607526,0.38166428,0.17531732],[-0.27549994,-0.8382197,-0.23044792],[0.27720013,0.10777386,0.38931534]],[[0.59602106,0.4931051,0.32372174],[0.26057044,-0.38513762,-0.25377673],[0.5239584,-0.60844016,-0.81516236]],[[-0.14618628,0.5511404,0.03257037],[-0.507615,-0.5938742,-0.18433402],[0.1405766,0.30189088,0.40384737]],[[0.683441,-0.12649333,0.1838069],[-0.07530575,-1.5419409,-0.14628708],[-0.3131361,-0.21054725,-1.0152699]],[[0.54563695,0.41037583,-0.5667209],[0.27594116,-0.34408566,0.07673677],[0.05612116,0.2613805,0.2782364]],[[-0.18580523,-0.23443171,0.2539113],[-0.22032824,-0.33988246,0.26845682],[-0.08496017,0.1464345,0.2081678]],[[0.48338115,0.3290803,-0.17630424],[-0.15116431,-1.0076884,-0.37554598],[0.47897416,0.12629706,1.1413]],[[-0.56012315,-0.16527249,-0.25457922],[0.16687028,1.0387771,0.6485804],[0.3251323,-0.48834637,-0.4849735]],[[0.69805586,-0.5177946,0.3641346],[-0.571902,1.3043618,-0.5279432],[-0.76316667,0.28352606,0.5489388]],[[1.2808713,0.7770489,0.11787289],[-0.7299889,-0.11740682,-0.022601],[-0.05397587,-0.4204368,0.5252781]],[[0.47005144,0.7351181,-0.10638901],[0.41841274,-0.30316615,-0.41939119],[-0.09935723,-0.05059246,-0.20005098]],[[0.56166345,0.16042244,-0.21918552],[-0.58513117,-0.12159419,-0.00208225],[-0.3569457,0.3016866,0.11292125]],[[-0.19453327,0.05771179,0.49587524],[-0.7059904,-0.07150852,-0.72072166],[0.0797845,-0.46644166,-0.32540146]],[[0.4047831,-0.8361217,0.02571795],[-0.06866717,-0.18578602,-0.01809985],[-0.13147792,-0.3287891,-0.06192441]],[[0.3932518,-0.05898901,-0.01544455],[0.37764654,0.80946136,-0.64825773],[0.7263728,-0.1496035,0.5965523]],[[0.58868694,-0.07772545,0.2702699],[0.3286201,-0.12455232,0.72229946],[0.66193694,-0.40314943,-0.6049827]],[[0.15490466,0.0790163,-0.29431176],[0.13548379,-0.4585114,0.325693],[0.8984879,0.35693708,0.58130825]],[[-0.25827298,1.0081819,0.71292245],[0.5003891,-0.24831942,0.44402534],[-0.3661488,-0.94714355,-1.1605774]],[[-0.57788295,-0.2193675,-0.21613039],[0.20407043,-0.08207346,-0.8275375],[-0.27110612,-0.49193633,0.96005726]],[[0.10024799,-0.28083974,0.03327747],[0.5289302,-0.5358489,-0.5377451],[0.03321321,-0.47960764,0.22873399]],[[0.6090003,-0.22203311,0.41142324],[-0.47119802,0.2107466,0.38376456],[0.04816642,-0.26131028,-0.4131246]],[[0.39337862,-0.39842346,-0.520124],[0.24224739,-0.60556316,-0.7164063],[0.06883961,-0.24349178,-0.17325647]],[[0.15957178,0.0908224,0.4014228],[-0.08994078,0.3044959,-0.25498646],[0.7323822,0.6140265,0.87483555]],[[-0.8186453,-0.32266617,0.0852558],[-0.58719563,-1.1645329,-0.55369395],[0.29357815,0.21708445,-0.55235636]],[[0.6909836,0.44096765,-0.5863495],[0.49566224,-0.2511729,-0.1377817],[-0.57162786,-0.22624145,-0.15158997]],[[0.17818071,0.7414978,0.2597051],[0.05637643,0.1015213,-0.600383],[-0.55664814,-0.131821,-0.09769516]],[[-0.2956643,-0.69748706,-0.09045231],[0.09473337,0.11972088,0.06039744],[-0.2988578,0.42204785,-0.0901257]],[[-0.74800766,-0.36905652,-0.7927181],[-0.7830767,0.02461081,-0.48229623],[0.42041138,-0.12755346,0.17736872]],[[-0.1166834,-0.6491379,0.41331354],[-0.56904894,-0.08686683,0.05495305],[0.11372701,0.480902,-0.6217819]]],[[[-1.95138144e+00,-3.53142731e-02,-4.45612997e-01],[-6.61472321e-01,-5.36171943e-02,-4.61290568e-01],[6.08084083e-01,5.32877594e-02,-2.54982620e-01]],[[-3.27214822e-02,4.81932312e-01,1.18701470e+00],[6.11389995e-01,1.84361055e-01,-4.17466640e-01],[9.09541398e-02,5.19658804e-01,4.53452915e-01]],[[-1.28615603e-01,-2.13283718e-01,-1.16975836e-01],[2.12834641e-01,2.36457348e-01,-6.24250114e-01],[-6.96309060e-02,1.29158899e-01,-5.19761026e-01]],[[-7.23119080e-01,-4.15512741e-01,-2.11584270e-01],[-5.01207769e-01,1.00673664e+00,4.89276230e-01],[-4.70817268e-01,6.12658933e-02,-3.69222909e-01]],[[-8.94557118e-01,-3.10563445e-01,-5.80785394e-01],[2.46288002e-01,1.62917033e-01,5.14172852e-01],[5.68964064e-01,-9.78494287e-01,-4.83733356e-01]],[[9.75359380e-02,-1.22473583e-01,-2.85169721e-01],[-3.22675794e-01,-4.39521670e-01,-6.03772640e-01],[-8.33871141e-02,-3.17209810e-01,2.42000684e-01]],[[-7.33887851e-02,-5.53621829e-01,3.69010381e-02],[-3.45516175e-01,-5.91681361e-01,-3.27353477e-01],[-3.99399363e-02,1.11532807e-01,8.02831233e-01]],[[1.34783053e+00,4.08623189e-01,-2.53248334e-01],[4.91286755e-01,1.35328192e-02,2.77334243e-01],[-5.34779251e-01,-1.38568640e-01,2.78700978e-01]],[[6.34547547e-02,6.50479078e-01,8.23392570e-02],[-9.03716907e-02,2.53675748e-02,-3.20071042e-01],[-6.59123421e-01,4.33011919e-01,-2.01148968e-02]],[[-4.88223374e-01,8.76686797e-02,4.08114940e-02],[6.11037910e-02,-4.38050419e-01,5.10003805e-01],[9.58591819e-01,-1.30651891e-01,-1.85997281e-02]],[[7.38362744e-02,-5.18548369e-01,-6.39519244e-02],[-4.76045072e-01,-4.27563399e-01,-7.60771275e-01],[3.63555640e-01,-5.94767928e-01,1.10442400e+00]],[[1.87034696e-01,-8.52006137e-01,3.71974468e-01],[-3.83464396e-01,-2.47029349e-01,1.61825065e-02],[-8.26955512e-02,-3.17373514e-01,-5.42173266e-01]],[[1.40968030e-02,1.48038745e-01,-1.16229832e-01],[-2.39001527e-01,2.28020564e-01,-4.44481581e-01],[-4.32632327e-01,-6.07848227e-01,-4.67280298e-01]],[[2.30089188e-01,3.87854010e-01,1.73868209e-01],[-1.46534219e-01,-1.37481824e-01,1.94740891e-02],[-1.78095579e+00,1.86246246e-01,-5.45462191e-01]],[[-5.79546750e-01,2.48257712e-01,-6.02766812e-01],[4.35355127e-01,8.15268695e-01,-1.17967322e-01],[-6.30107820e-02,-3.93636972e-01,3.00807953e-01]],[[-3.32165509e-01,-1.33447617e-01,-3.88576448e-01],[3.57200444e-01,3.55643779e-01,1.96641341e-01],[3.26390922e-01,1.97860580e-02,-9.72058594e-01]],[[7.68835068e-01,-6.54250920e-01,4.80657518e-01],[5.67117512e-01,4.24815476e-01,2.93651730e-01],[1.35204959e+00,-2.45922536e-01,4.34941828e-01]],[[9.25367028e-02,8.05485900e-03,1.81562215e-01],[-4.58917707e-01,3.45895559e-01,2.41738707e-01],[-1.60645723e+00,-1.75902545e-01,2.25259990e-01]],[[8.06524232e-02,1.80403560e-01,6.75317228e-01],[1.49968117e-01,-1.09927908e-01,-1.62559226e-01],[-3.91388386e-01,5.24677634e-01,2.26059958e-01]],[[4.30916637e-01,2.71923333e-01,8.60539079e-01],[3.51739138e-01,4.49953198e-01,9.81391251e-01],[-9.19201151e-02,-2.77235755e-03,1.12169288e-01]],[[2.58823842e-01,-1.43996492e-01,1.25087130e+00],[-1.92480087e-02,-4.28758472e-01,4.32047509e-02],[-6.11154377e-01,2.85474569e-01,1.34136453e-01]],[[1.07521486e+00,-1.65449426e-01,3.04549634e-01],[4.20869291e-01,5.34772081e-03,-3.76286842e-02],[2.25134805e-01,2.51246095e-01,-2.18565479e-01]],[[5.15453219e-01,1.61690027e-01,3.57044697e-01],[8.26454759e-02,1.90989852e-01,3.26628685e-01],[-4.71664429e-01,3.39085720e-02,2.61175096e-01]],[[-4.99644011e-01,3.28845739e-01,-7.27535337e-02],[3.05795491e-01,-1.08540552e-02,3.63353699e-01],[-9.93884444e-01,3.72558117e-01,4.59596425e-01]],[[7.83227026e-01,2.64018357e-01,4.35987055e-01],[6.70060575e-01,-2.56562442e-01,6.31654620e-01],[1.05469978e+00,-2.18920529e-01,-2.28367403e-01]],[[1.59314886e-01,4.02576208e-01,5.65976277e-02],[1.84918150e-01,1.98422194e-01,-6.77455068e-02],[9.71419290e-02,2.40607247e-01,-4.07588780e-01]],[[-8.23372483e-01,-2.61967152e-01,6.74648210e-02],[2.48087972e-01,-3.32321435e-01,7.61052370e-02],[-4.73876059e-01,2.36241594e-01,-4.88194861e-02]],[[-8.31796408e-01,-4.36141267e-02,-4.14752722e-01],[-1.13889091e-01,-1.60126120e-01,2.26139113e-01],[-3.41105014e-01,4.54437256e-01,4.62299824e-01]],[[1.55052081e-01,-1.88724268e-02,3.30424190e-01],[-7.80875862e-01,-3.94310467e-02,5.55676259e-02],[4.22747016e-01,-2.11918578e-01,-4.24767524e-01]],[[4.92707253e-01,3.64238411e-01,1.92559436e-02],[-1.03528768e-01,2.36791447e-01,-1.62654012e-01],[-5.68739831e-01,2.55604953e-01,-1.09464973e-01]],[[-7.33454347e-01,4.95864749e-01,5.72742999e-01],[-3.09416294e-01,4.08730768e-02,3.57128590e-01],[-6.49583220e-01,-6.62931725e-02,8.29529092e-02]],[[1.50755182e-01,-2.25918517e-01,1.03902601e-01],[6.50762737e-01,-1.97168782e-01,-2.80983329e-01],[-5.93105316e-01,-1.28149474e+00,-3.81950557e-01]],[[4.22517955e-01,4.60478127e-01,5.51381588e-01],[3.06708783e-01,-4.03679311e-01,6.56210423e-01],[-4.55646724e-01,-5.97298257e-02,-7.83346826e-04]],[[7.31525958e-01,3.81487273e-02,-2.91222185e-01],[3.29751000e-02,-1.94853306e-01,-3.08487006e-02],[-6.30285740e-01,-8.74083042e-02,-4.93079377e-03]],[[1.57251835e-01,1.63098127e-01,-4.72049057e-01],[3.24987680e-01,-2.83619761e-01,-9.62147489e-03],[-1.04311600e-01,-5.19614100e-01,-1.89243883e-01]],[[-1.47429422e-01,-6.97931468e-01,-4.48473543e-01],[-2.50531375e-01,-2.79857427e-01,-2.02996328e-01],[-3.22159767e-01,4.01236355e-01,-9.04781669e-02]],[[4.57688957e-01,4.92140174e-01,-5.65403521e-01],[6.71054125e-01,-1.80676818e-01,-1.04767375e-01],[-2.34471276e-01,6.80567920e-01,-2.95701355e-01]],[[5.67788482e-01,-8.40078473e-01,-4.27655309e-01],[5.15610456e-01,-1.43058985e-01,1.19855240e-01],[1.36588776e+00,1.87380046e-01,4.07116681e-01]],[[-4.07560021e-01,2.98769236e-01,1.45845950e-01],[-3.22544217e-01,3.20536733e-01,-6.01069331e-01],[7.19784573e-02,5.91837466e-01,-1.46754742e-01]],[[6.77713454e-02,5.45492172e-01,-1.94034609e-03],[-1.97263420e-01,1.63331956e-01,2.38321409e-01],[-2.17568532e-01,-3.55524361e-01,8.05568039e-01]]],[[[-5.11851069e-03,3.77450645e-01,3.33727390e-01],[-4.05309200e-01,-1.60685435e-01,-2.49451011e-01],[2.84474999e-01,1.23812724e-02,-9.81268957e-02]],[[-1.00229859e+00,2.36921713e-01,-2.29718119e-01],[-1.38300598e-01,3.90331149e-01,-5.57963610e-01],[2.12779313e-01,-1.73552185e-01,-3.83072466e-01]],[[-8.57702792e-01,4.36028540e-01,-9.76581633e-01],[-6.73210859e-01,-4.58474994e-01,6.36899769e-01],[1.90075830e-01,4.13881838e-02,-4.81884331e-01]],[[6.52405739e-01,-4.89686489e-01,-4.98745888e-01],[3.87001447e-02,-3.83601308e-01,4.98893291e-01],[-2.36480713e-01,-3.27852547e-01,-3.17356497e-01]],[[-2.45373055e-01,5.17752990e-02,1.20020640e+00],[-1.96496516e-01,1.56255692e-01,-2.71129012e-01],[-2.08206922e-01,-5.81233064e-03,1.21275246e+00]],[[1.39994562e-01,-2.16018662e-01,-5.47971129e-01],[-2.84902155e-01,1.39471471e-01,-1.28828317e-01],[6.00679398e-01,2.74609216e-02,-2.46436775e-01]],[[-9.38557148e-01,-6.93620443e-02,9.42982510e-02],[-8.77606630e-01,-3.63361329e-01,-9.36947703e-01],[1.24724247e-01,1.60072878e-01,-2.85068780e-01]],[[4.75211233e-01,-2.04335123e-01,5.90182357e-02],[7.41418064e-01,4.66522753e-01,1.74888328e-01],[8.73277664e-01,1.41470268e-01,5.32321751e-01]],[[1.75918624e-01,5.14055610e-01,4.38589990e-01],[1.35607183e-01,-2.43688077e-01,-1.14665389e+00],[-5.90185933e-02,-7.05488622e-01,1.65454333e-03]],[[-4.86672580e-01,-4.89178210e-01,2.72594512e-01],[5.53016961e-01,2.01369345e-01,5.39292812e-01],[-2.03462943e-01,-5.42188585e-01,-5.00312090e-01]],[[-3.06347851e-03,-3.50663997e-03,1.87393233e-01],[-4.18928891e-01,5.78538895e-01,9.31874365e-02],[-7.92276740e-01,1.13861538e-01,1.92502812e-01]],[[-2.47007146e-01,6.67524710e-02,-2.20888004e-01],[-6.66758269e-02,-9.53671873e-01,-6.50125444e-01],[1.64288605e-04,-4.59389389e-01,2.54025385e-02]],[[-1.07798472e-01,-1.55717343e-01,-2.89332476e-02],[4.75288898e-01,-5.53906798e-01,8.14318657e-01],[1.49215415e-01,5.29029779e-02,1.20860434e+00]],[[1.25063643e-01,-1.81598440e-01,-8.57714564e-02],[3.92436355e-01,-1.41856233e-02,6.97281897e-01],[-5.15500903e-01,2.65269905e-01,-2.95438111e-01]],[[-3.71486992e-01,6.25076368e-02,9.30654347e-01],[2.59205312e-01,-2.32675403e-01,-4.84896511e-01],[-4.29534882e-01,6.14406228e-01,-3.55034381e-01]],[[3.78138095e-01,-6.13357067e-01,2.39900976e-01],[2.96919256e-01,4.76108581e-01,3.76450837e-01],[3.88622344e-01,-3.89822185e-01,4.71801668e-01]],[[4.35241461e-01,3.66345882e-01,5.84786609e-02],[3.79125059e-01,1.50400564e-01,2.56609041e-02],[1.02059685e-01,-5.88801242e-02,-4.16280508e-01]],[[-4.82504278e-01,7.64246225e-01,1.91834107e-01],[2.23704413e-01,-6.76671565e-01,2.23960698e-01],[1.38094455e-01,3.57407480e-01,8.83019745e-01]],[[3.45634550e-01,1.86573938e-01,-1.53048718e+00],[-6.47696853e-01,3.60320985e-01,-2.10069403e-01],[6.01095855e-01,3.32261473e-01,-6.00790381e-01]],[[6.65431798e-01,3.44572872e-01,1.43107101e-01],[1.86530024e-01,1.04681589e-01,-2.94636428e-01],[-2.08374515e-01,3.54862988e-01,9.52960074e-01]],[[4.29627270e-01,-2.04713136e-01,3.70003609e-03],[-1.62252739e-01,-1.63621604e-01,5.63292325e-01],[-1.44923955e-01,-2.15824857e-01,2.79529959e-01]],[[8.77226591e-01,5.14212288e-02,1.63558766e-01],[8.62792432e-02,3.64067465e-01,6.32324591e-02],[-2.87758738e-01,3.89290512e-01,1.23024628e-01]],[[1.14763781e-01,-1.89301252e-01,-6.25126421e-01],[-4.21713233e-01,9.90234390e-02,-5.15302360e-01],[-2.79421061e-01,3.79200578e-02,-1.38988316e+00]],[[1.93796679e-01,2.15909451e-01,-2.16033682e-01],[-2.80253589e-01,8.05382788e-01,-7.84899473e-01],[-5.21930397e-01,6.17383242e-01,7.83126533e-01]],[[3.28324288e-01,-1.62669390e-01,-3.46312344e-01],[2.61233952e-02,4.62257653e-01,2.40267754e-01],[1.84016794e-01,-3.01273227e-01,-5.53452134e-01]],[[6.05316103e-01,8.79511535e-02,-3.50149393e-01],[1.93789721e-01,-3.35362703e-01,-1.70937657e-01],[-5.52736223e-01,-5.61839402e-01,-8.75173450e-01]],[[4.39989686e-01,1.13611042e-01,8.90986100e-02],[-4.04582433e-02,2.53495187e-01,-3.88558567e-01],[6.25039697e-01,-9.79334831e-01,2.61527210e-01]],[[8.04518580e-01,-4.76878077e-01,-4.02857751e-01],[1.53495386e-01,4.00287271e-01,1.07920003e+00],[3.96059722e-01,3.36384386e-01,4.05256897e-01]],[[-3.53547961e-01,-2.69692391e-01,8.97947609e-01],[1.96808994e-01,-1.13331340e-01,5.13785362e-01],[9.31719020e-02,-4.08225387e-01,2.41515841e-02]],[[-3.75714213e-01,1.74871851e-02,-2.55440921e-01],[7.33769119e-01,-5.91310978e-01,-1.22804868e+00],[3.13686490e-01,-1.50593340e-01,-2.71445453e-01]],[[-2.34713644e-01,1.04809143e-01,3.87570560e-01],[-4.51907784e-01,6.70721769e-01,-7.79417515e-01],[-9.01541859e-02,-1.94519386e-01,-2.30928585e-01]],[[-2.29041934e-01,-9.50614661e-02,3.65041435e-01],[6.24890864e-01,-5.59796154e-01,1.49865508e-01],[5.94657719e-01,2.48228520e-01,-4.72197890e-01]],[[-6.39830977e-02,1.11223078e+00,-1.99890107e-01],[4.09965813e-01,-3.50762010e-02,1.13447869e+00],[-2.17305496e-01,2.41173550e-01,1.89887047e-01]],[[2.04904363e-01,-4.16224562e-02,7.78917372e-01],[1.63469732e-01,-7.28012919e-01,-2.69906223e-01],[-2.59984612e-01,-1.95528001e-01,5.61278343e-01]],[[-1.42784253e-01,-4.11888510e-01,7.86375403e-01],[1.75751984e-01,-6.00266978e-02,3.92498970e-01],[-2.21793968e-02,-7.38070130e-01,6.21427238e-01]],[[5.00833452e-01,-5.89961171e-01,7.82852352e-01],[-1.54137820e-01,-3.96219850e-01,1.49969682e-01],[-1.10980463e+00,-1.61959961e-01,-1.82444483e-01]],[[-3.23724031e-01,-3.94545883e-01,5.41369200e-01],[-2.41292894e-01,-1.38272330e-01,-1.96571186e-01],[-4.88416374e-01,5.78442872e-01,-7.61529386e-01]],[[-6.08103611e-02,-2.40503207e-01,-3.75307381e-01],[1.22369722e-01,-2.17141837e-01,-6.47891998e-01],[8.05405900e-02,2.13645816e-01,1.98020473e-01]],[[-2.18496040e-01,5.55782378e-01,5.30253589e-01],[-1.03926623e+00,1.32370338e-01,-2.00147212e-01],[6.86323121e-02,-3.43177579e-02,6.39821112e-01]],[[-2.26147950e-01,6.23070486e-02,3.86770427e-01],[2.31429324e-01,-4.38826889e-01,-1.84120730e-01],[-1.48639143e-01,-6.22478902e-01,6.37045801e-01]]],[[[-0.32057393,0.48127294,0.7921301],[-0.50106364,-0.37172526,0.20014693],[1.0592903,0.628085,-0.778133]],[[-0.8350821,0.04369668,0.1360083],[0.33034098,-0.5856764,-0.19452907],[-0.1536473,0.30432847,-0.28498265]],[[0.21340829,-0.13739301,-0.626666],[-0.26841274,-0.06780782,0.24714816],[-0.02771953,0.06142968,-0.21344796]],[[-0.14225529,-0.4137714,0.4759821],[0.19542769,-0.47023112,-0.5324625],[0.28854093,-0.29795793,0.00397929]],[[0.30816925,-0.33861342,-0.75590926],[-0.09618121,0.5428758,0.41271546],[-0.555407,-0.42413905,0.16565172]],[[-0.73163575,0.00468567,-0.05380987],[-1.0745909,-0.51702595,-0.9311388],[0.20508938,1.1839375,-0.40826592]],[[0.17759791,-0.53451115,-0.22417656],[1.0109937,-0.57462823,0.23353657],[-0.73670185,0.14394413,0.4735296]],[[0.531431,-0.54107565,0.28506967],[-0.3444799,0.12913974,-0.26902053],[0.23266634,0.49908647,0.5199798]],[[0.29095012,-0.23932353,0.06798673],[-0.3252704,0.06316344,-0.02161698],[0.5038485,0.17916544,0.1588459]],[[-0.49409157,-0.4751864,-0.46300307],[-0.44502723,0.12898943,-0.01330722],[-0.5160724,-0.68882257,0.45784754]],[[0.4102395,-0.33728904,0.85974324],[-0.52931374,-0.1420324,0.13546103],[0.8328249,-0.00779751,0.69834244]],[[-0.13218091,-0.28659034,0.5586661],[-0.6906351,0.18902573,0.36832127],[0.3019043,-0.41243136,0.20260061]],[[0.2859215,0.57147336,0.4836285],[0.05326753,-0.0987342,0.35218844],[-0.15387508,-0.09863772,-0.01492346]],[[-0.06243335,-0.5661319,-0.34560153],[0.64269036,-0.6872679,0.42706373],[0.68278235,-0.4700272,-0.32586855]],[[0.53617287,-0.2889359,-0.02656382],[0.3065854,-0.31970176,0.17844664],[-0.28576344,0.48151854,0.09637637]],[[-0.2606406,-0.16500074,0.19967432],[-0.16401403,0.23431608,-0.7690197],[-0.46227005,-0.3824779,0.25618258]],[[-0.7132731,0.31157902,-0.32327208],[-0.679808,0.36148876,-0.12814873],[0.38892588,-0.62332803,0.48038778]],[[-0.20709415,0.38604534,0.21159367],[0.6277198,-0.5465007,0.28554836],[-0.06276698,0.1894167,-0.3841785]],[[-0.6571471,0.13822295,0.13112378],[-0.36196578,0.19235462,-0.11745107],[0.3242107,-0.20410293,0.06942195]],[[-0.01404161,-0.5267923,-0.2182335],[-0.1654929,-0.47746992,0.00231044],[-0.18928964,0.5533064,-0.6207964]],[[-0.61811846,-0.03696335,0.16599914],[-0.27078635,0.6305948,0.42998043],[0.13523988,-0.78510976,-0.14491057]],[[-0.21212272,-0.39075404,0.4833065],[-0.9518733,-0.32190633,0.2674024],[0.34385177,0.15228032,-0.13966517]],[[-0.74261856,0.47722518,-0.03025851],[-0.29980505,0.2971787,0.05451442],[0.0275174,0.29444468,-0.4274594]],[[-0.07302048,0.05765961,-0.3804249],[-0.31374037,0.07472562,-0.24762185],[0.61957395,-0.07180076,-0.8734377]],[[-0.25787413,-0.3671486,0.20110495],[-0.35487926,-0.3334408,-0.2595482],[-0.37672204,-0.12584879,-0.02109473]],[[-0.7815018,-0.00663437,-0.09478676],[0.01142317,0.30415705,-0.24620728],[0.24452035,0.59034765,0.36044604]],[[-0.44807145,0.5546522,0.69528705],[-0.15689613,-0.20073585,0.05189526],[-0.34003595,-0.07404114,-0.48127833]],[[0.34275094,-0.07729276,-0.45592597],[-0.33170062,-0.17782019,-0.2068065],[0.2831229,0.40096942,0.6251401]],[[0.37283394,0.19636124,1.0366828],[0.35837656,0.03303956,0.5615019],[0.38274595,-0.45080993,0.26776084]],[[0.08627263,-0.21417767,-0.73673487],[-0.01730959,-0.0841051,0.13940662],[0.0717696,0.61843514,0.22665201]],[[0.96772647,0.4443506,-0.05496972],[0.87388414,-0.45656574,-0.6852613],[0.7225186,0.24664955,-0.3470612]],[[-0.20134062,0.28035602,0.07656232],[0.16857103,-0.05843551,0.02353913],[0.43742624,-0.72698045,0.6864028]],[[-0.5845986,-0.5659726,-0.2100087],[0.36376894,0.22221377,-0.06353877],[-0.05765058,0.12544951,0.10957355]],[[-0.00544495,-0.446207,0.7871389],[-0.4940833,-0.02581003,-0.3618002],[0.44729155,-1.7471527,0.15924828]],[[0.10670286,-0.63102233,0.19685778],[-0.69691175,0.7663174,-0.7154208],[-0.20084639,-0.12778327,1.5313358]],[[0.6024284,1.2310224,-0.24922624],[-0.43424863,-0.39106086,-0.09398185],[0.23710273,-0.33362076,0.6375694]],[[0.6095869,0.5223068,0.12847665],[-0.05789297,-0.4098005,-0.069596],[0.22797194,-0.74972016,-0.1601936]],[[-0.09029771,-1.0111716,0.4087255],[0.14212763,0.29607973,-0.7109833],[-1.4342351,-0.31746033,-0.5645867]],[[0.8244864,-0.2962702,0.6097062],[-0.4054325,-0.7654451,-0.22215152],[-0.55307215,0.01942207,-0.21280104]],[[-0.5267938,0.26709127,-0.45423946],[-0.12865017,0.17677505,-0.4129848],[0.33423108,-0.17203753,0.05141009]]],[[[1.61734009e+00,-1.14838041e-01,-4.49140340e-01],[2.02679515e-01,3.00842464e-01,-1.12471633e-01],[2.46619076e-01,4.85331386e-01,-2.56520450e-01]],[[-5.76808035e-01,3.91263634e-01,2.73924410e-01],[-1.00940466e+00,-6.99599266e-01,-3.17919165e-01],[-3.29365522e-01,-1.08682573e+00,3.41671258e-01]],[[5.65080762e-01,-4.94071782e-01,-3.19443673e-01],[-6.87343627e-02,1.31215855e-01,3.04978162e-01],[-2.10081082e-04,1.89764053e-02,-1.67566478e-01]],[[-4.16589379e-01,4.05841358e-02,1.87385142e-01],[9.67049077e-02,1.88301280e-01,-6.56510413e-01],[1.18341736e-01,5.83960295e-01,-3.75011355e-01]],[[-2.32528493e-01,-2.24846713e-02,-1.41501248e-01],[9.45636690e-01,2.85433590e-01,8.21929350e-02],[9.08571005e-01,-7.39691317e-01,5.24960160e-01]],[[-4.56337601e-01,-5.08086860e-01,-2.01173976e-01],[-5.31273007e-01,5.10257259e-02,1.03608930e+00],[3.97805780e-01,6.37536764e-01,2.66702473e-01]],[[-9.84524250e-01,-1.56235084e-01,1.01309806e-01],[3.27909365e-02,-1.47641584e-01,-5.87275028e-01],[8.17374051e-01,-3.19929957e-01,-6.12605453e-01]],[[-6.52957082e-01,2.59046227e-01,7.97139227e-01],[-9.94958282e-02,4.54255730e-01,4.00304198e-01],[-3.46643656e-01,5.37533462e-01,-2.93051481e-01]],[[1.95701584e-01,4.61963326e-01,1.01822786e-01],[-6.01513863e-01,-1.03409803e+00,-2.90306769e-02],[-5.43642044e-01,-2.37469509e-01,-2.75474489e-01]],[[-7.69555032e-01,3.64582986e-01,1.00081362e-01],[1.97588429e-01,1.78822994e-01,-7.68736899e-02],[1.05518789e-03,1.86270237e-01,-1.50004044e-01]],[[-3.52719367e-01,1.65323183e-01,9.04975608e-02],[-8.98519456e-02,-2.61867613e-01,5.42057753e-01],[4.24820691e-01,-3.60448331e-01,-3.44871394e-02]],[[1.07692325e+00,-2.11277559e-01,7.16012567e-02],[2.05282435e-01,-2.85328448e-01,3.38517018e-02],[-4.55096930e-01,-2.60051429e-01,3.78876865e-01]],[[-5.45406997e-01,-2.01727301e-01,-6.04762971e-01],[5.14230072e-01,-6.90163791e-01,7.65970230e-01],[6.36966646e-01,-6.09393954e-01,5.89231662e-02]],[[-2.92210370e-01,-2.55249977e-01,5.49844086e-01],[-4.84601647e-01,-2.61553019e-01,1.12557421e-02],[2.52797723e-01,-3.07967722e-01,-7.55000412e-01]],[[9.19248521e-01,4.24327940e-01,4.12770286e-02],[-3.54415506e-01,-3.30701292e-01,-6.21597946e-01],[-1.04827002e-01,-7.11925685e-01,2.43620962e-01]],[[1.15475915e-01,-7.15647414e-02,5.77222705e-01],[5.80904424e-01,-3.28314453e-01,-6.64207935e-01],[-7.33095467e-01,5.19213937e-02,-4.30541217e-01]],[[-2.07538873e-01,2.54654467e-01,5.21439910e-02],[-7.97100902e-01,5.30928075e-01,-4.81240332e-01],[7.04245493e-02,1.33390933e-01,2.50584811e-01]],[[-6.64525151e-01,-1.76937655e-01,-3.64606738e-01],[-4.77653146e-01,4.46024656e-01,5.38305223e-01],[-8.20155203e-01,-3.06513727e-01,-2.92145163e-01]],[[-3.74347448e-01,3.64892989e-01,4.45089340e-02],[-1.08425069e+00,-2.03151748e-01,-1.42824933e-01],[-4.81627673e-01,2.09600300e-01,-1.41540334e-01]],[[-1.05690256e-01,-1.72056660e-01,-1.44753337e-01],[-5.82157373e-02,-8.31891075e-02,3.51638198e-01],[-4.47725683e-01,5.29997885e-01,-9.48509872e-01]],[[-1.43295705e-01,-1.42493665e-01,1.48418650e-03],[-3.22893262e-01,-9.70917270e-02,-1.06634056e-04],[-5.48445761e-01,-1.22192197e-01,1.19758427e-01]],[[2.79990673e+00,2.08021864e-01,5.01127899e-01],[1.14319718e+00,5.59349179e-01,-6.75364360e-02],[1.10963620e-01,6.67478964e-02,-3.96308243e-01]],[[-1.67182952e-01,1.71256736e-01,7.87570477e-02],[-8.60321075e-02,-1.78988054e-01,3.47869456e-01],[3.42493504e-01,3.23814780e-01,2.30804622e-01]],[[7.13812560e-02,-2.10237011e-01,4.40327883e-01],[2.63161302e-01,-1.43130705e-01,-3.51753086e-02],[-5.06826282e-01,-7.55303204e-02,-4.52103794e-01]],[[-1.42995967e-02,-1.07819088e-01,-7.20670879e-01],[-3.39731872e-01,1.40296854e-02,-2.53111333e-01],[-6.89081967e-01,2.21704066e-01,-2.55370498e-01]],[[7.98865259e-01,3.33394289e-01,-2.18718261e-01],[1.89883038e-01,6.20884359e-01,7.03062773e-01],[-2.26601064e-01,-1.42649949e-01,1.88721389e-01]],[[6.78196371e-01,-2.94871509e-01,1.33821666e-01],[2.95686722e-02,-1.21665232e-01,9.02671367e-02],[-3.98633838e-01,-1.62980154e-01,-4.36248071e-02]],[[3.00527066e-01,-3.43163997e-01,-2.54522741e-01],[1.43857881e-01,8.91035870e-02,8.11714754e-02],[-8.14516425e-01,-3.21071953e-01,-4.12893653e-01]],[[-2.38900483e-01,1.73989579e-01,7.64437318e-01],[1.86876819e-01,-8.55067223e-02,-7.14658260e-01],[-2.20880792e-01,5.38473763e-02,3.48950103e-02]],[[-2.87929654e-01,2.56195754e-01,-3.04997295e-01],[-1.52085423e-01,5.62779665e-01,-1.01807125e-01],[-3.35797638e-01,-4.94052321e-02,7.69823611e-01]],[[-8.85751605e-01,-3.97341661e-02,5.89237154e-01],[-2.93699205e-01,-3.68156165e-01,4.21072692e-01],[-2.49336883e-01,-1.80402547e-01,6.85328364e-01]],[[1.83292896e-01,-9.36522007e-01,-2.34595194e-01],[-1.89473853e-01,2.89177239e-01,3.19117099e-01],[2.03595281e-01,-5.50776362e-01,-2.99911261e-01]],[[-7.85570502e-01,5.04994169e-02,7.19722807e-01],[-1.44616902e-01,2.05191404e-01,3.68143052e-01],[1.69145226e-01,7.86887705e-01,-1.42142534e-01]],[[-2.28178501e-01,-6.91841960e-01,-5.51680267e-01],[-2.24008054e-01,2.65543222e-01,-1.06778264e+00],[-4.64908928e-01,-1.26889721e-01,1.96428657e-01]],[[4.33143139e-01,-2.72287548e-01,5.95820606e-01],[-1.36281297e-01,6.12523139e-01,-4.94322866e-01],[5.21611869e-02,1.14164688e-01,5.24308026e-01]],[[7.83473670e-01,-1.52832195e-01,-8.73274133e-02],[-2.83857018e-01,2.37636134e-01,-1.29461838e-02],[-6.43388107e-02,-4.95547891e-01,-3.39420199e-01]],[[-5.01180291e-01,3.59908760e-01,4.28256750e-01],[9.76974741e-02,-4.26841766e-01,8.91304836e-02],[-6.48894131e-01,-3.36071044e-01,-2.66685843e-01]],[[5.76224387e-01,1.69820800e-01,-6.53727055e-02],[2.44861588e-01,4.12008345e-01,-7.34486520e-01],[2.90023267e-01,-2.34081239e-01,5.62268972e-01]],[[1.89244047e-01,2.85778016e-01,-7.47780502e-02],[-3.98015112e-01,-1.75518811e-01,4.30301934e-01],[2.45281249e-01,3.33774537e-01,7.03208670e-02]],[[-9.43441808e-01,-1.94948148e-02,-2.61213690e-01],[-4.65070084e-02,2.96963722e-01,-3.61284733e-01],[1.19633429e-01,-4.15274799e-01,-3.46090585e-01]]],[[[1.00860171e-01,-2.40390867e-01,-2.06436291e-01],[1.49847165e-01,1.29264429e-01,1.33979082e-01],[5.34262918e-02,4.49309081e-01,6.56260550e-01]],[[2.53706992e-01,3.08016121e-01,-7.75962025e-02],[-3.19974899e-01,-1.19100082e+00,5.88966131e-01],[-1.15586556e-01,4.89629894e-01,-3.08796555e-01]],[[6.94318414e-01,2.73533076e-01,3.94319266e-01],[3.73087347e-01,-4.91358005e-02,6.06686771e-02],[-2.48837575e-01,-2.58526057e-02,-1.66287273e-01]],[[5.16920984e-01,-1.20097935e-01,-6.60666823e-02],[2.06386343e-01,-5.51049054e-01,-6.10004008e-01],[-1.43948808e-01,5.27445674e-01,-2.33507991e-01]],[[-1.60685152e-01,2.35182494e-01,-3.26961815e-01],[-3.01131099e-01,3.21941018e-01,-7.24017203e-01],[-6.19922936e-01,6.99700058e-01,-1.28578496e+00]],[[1.40200436e-01,2.74488270e-01,3.00092041e-01],[-2.59320140e-01,-3.77929956e-01,2.27601737e-01],[1.40764862e-01,-4.94325459e-01,-5.05851388e-01]],[[2.78062433e-01,-4.06010509e-01,-6.13084808e-03],[1.17114052e-01,-2.48769537e-01,-4.71994281e-01],[5.25295734e-01,-2.70041041e-02,-6.55163452e-02]],[[6.57382250e-01,1.03099771e-01,1.43822744e-01],[2.57293642e-01,2.05132082e-01,-3.85786295e-01],[-4.98273015e-01,-2.26845339e-01,1.11277592e+00]],[[-3.69648665e-01,-6.13242626e-01,-4.59410816e-01],[-8.94120455e-01,1.23310767e-01,-2.01187000e-01],[1.20630942e-01,-4.97758463e-02,-4.14306581e-01]],[[4.06720400e-01,7.42347777e-01,9.13650692e-02],[-7.91266095e-03,6.31320119e-01,-3.82018894e-01],[2.72133082e-01,1.38808787e-01,-3.20734501e-01]],[[-3.29673707e-01,-2.73059346e-02,-3.16542000e-01],[5.04684985e-01,2.38482535e-01,-3.94236475e-01],[-1.24661255e+00,-2.27463350e-01,-2.02249199e-01]],[[2.99714148e-01,7.65825212e-02,-2.83599347e-02],[5.60732186e-01,3.71094853e-01,-7.09657669e-01],[-4.59304839e-01,-5.59783757e-01,7.01333106e-01]],[[-2.04073802e-01,-5.73726118e-01,2.05665857e-01],[-2.91053146e-01,-1.54975086e-01,-9.06107843e-01],[-1.47010148e-01,2.50082552e-01,1.34367704e-01]],[[-6.44371331e-01,-3.96543026e-01,1.07195711e+00],[5.75075686e-01,-1.49294302e-01,-1.15917340e-01],[3.93327862e-01,-3.07970345e-01,1.06995210e-01]],[[6.10055625e-01,9.00474116e-02,2.12805972e-01],[-8.74010846e-02,-2.02239498e-01,2.82340705e-01],[-1.14959943e+00,-9.22432914e-02,4.01120424e-01]],[[-6.36559844e-01,4.92666066e-02,-4.67725724e-01],[-3.38008344e-01,-3.30233186e-01,-3.01290751e-01],[6.13975013e-03,-6.08625829e-01,-7.47681320e-01]],[[-1.21335459e+00,-2.49201506e-01,4.70094442e-01],[-1.06309378e+00,-5.14388263e-01,-3.95741612e-02],[2.21987545e-01,5.51686808e-02,-4.66010809e-01]],[[-6.93523884e-01,-2.04346955e-01,1.48656247e-02],[-4.00666296e-01,-4.82375681e-01,1.01161666e-01],[4.85308647e-01,-7.30794435e-03,6.97794259e-02]],[[5.44703722e-01,3.70247692e-01,6.42843470e-02],[5.46566769e-02,3.16445269e-02,-4.01390791e-02],[-4.58206892e-01,5.69090962e-01,1.70241017e-02]],[[4.90610093e-01,-1.25693277e-01,-7.82398432e-02],[1.26092419e-01,-3.82057577e-01,1.65130943e-01],[-1.09797359e-01,-4.86658871e-01,4.55541432e-01]],[[-5.78925431e-01,1.90718636e-01,9.83891308e-01],[1.74183920e-01,1.07138835e-01,2.18132123e-01],[1.63386762e-01,4.94732350e-01,1.22774072e-01]],[[-1.87488005e-01,-2.11622491e-01,-4.63737041e-01],[-2.52744466e-01,1.82059959e-01,-6.40743196e-01],[2.89451391e-01,-2.00128064e-01,-1.96605265e-01]],[[-4.98953089e-02,-1.37443691e-01,1.03907335e+00],[-7.33789563e-01,-3.07671875e-01,2.67478436e-01],[6.77073076e-02,2.27507845e-01,1.10462219e-01]],[[-1.29286170e+00,-8.58273685e-01,2.92836100e-01],[1.02795804e+00,-6.87610134e-02,4.46060777e-01],[-2.69497305e-01,-3.14267337e-01,4.95184392e-01]],[[-1.85915500e-01,-7.33863413e-01,6.84350073e-01],[-1.92909226e-01,-2.07036689e-01,4.24158692e-01],[4.05250698e-01,-2.18467489e-01,-1.86528057e-01]],[[3.90627623e-01,6.74346864e-01,1.65216208e-01],[-2.48045653e-01,1.23583086e-01,-3.48470837e-01],[1.24200130e+00,-3.73688340e-01,-7.50984311e-01]],[[8.53551388e-01,-8.47996950e-01,-7.61734962e-01],[-6.41514957e-01,3.25262584e-02,-1.19002454e-01],[-2.37232335e-02,-3.09668869e-01,-1.58258341e-02]],[[-2.97769427e-01,2.93022871e-01,-3.24457049e-01],[-1.64763525e-01,-3.75193447e-01,-3.97772521e-01],[3.70118797e-01,6.28294870e-02,7.79163718e-01]],[[2.56423116e-01,-3.90087873e-01,-5.41911900e-01],[-1.90978557e-01,-1.61314402e-02,9.48811620e-02],[1.20196737e-01,7.27423251e-01,-6.91013813e-01]],[[4.69950261e-03,7.29241133e-01,-1.98549554e-01],[1.95156056e-02,-2.00917929e-01,-2.17379242e-01],[2.98188686e-01,6.05272412e-01,1.25450492e-01]],[[6.72376677e-02,-2.86604077e-01,1.92449123e-01],[-1.38409805e+00,2.18003467e-02,5.27253330e-01],[3.08025926e-01,-1.47854850e-01,2.97343165e-01]],[[1.35307550e-01,-5.21705925e-01,-5.64323545e-01],[2.81850010e-01,5.28032705e-02,-6.72520459e-01],[1.39566466e-01,-5.73251009e-01,-1.48034126e-01]],[[7.54971988e-04,1.21798366e-01,4.88908470e-01],[3.81742656e-01,2.87844650e-02,8.63750279e-02],[-3.65633368e-01,7.11751163e-01,3.37526172e-01]],[[1.53856754e-01,9.48303819e-01,-1.81834549e-01],[-1.33604705e-01,1.26859590e-01,1.85263172e-01],[4.56026733e-01,8.36337507e-01,-1.97340757e-01]],[[3.34573328e-01,1.70396566e-01,-8.35757673e-01],[-9.51839332e-03,-6.22968733e-01,-1.88637018e-01],[-5.16888559e-01,8.90954807e-02,-7.89933026e-01]],[[-1.54233992e-01,1.62870690e-01,1.76866129e-01],[-1.80088803e-01,-7.19903171e-01,2.52553880e-01],[-6.35493875e-01,-5.22329882e-02,-2.23233357e-01]],[[-6.80267066e-02,-8.00985813e-01,3.23801875e-01],[4.21054423e-01,-7.82713220e-02,8.93486977e-01],[6.76034749e-01,1.59521222e-01,3.59713882e-01]],[[7.14856982e-01,1.69721663e-01,1.35735705e-01],[7.86560550e-02,1.87706560e-01,1.01170883e-01],[2.93092072e-01,-1.94798631e-03,-1.29291773e+00]],[[-8.50416049e-02,1.69862866e-01,1.34005398e-02],[1.45633662e+00,1.45070419e-01,-2.54939497e-01],[1.95306242e-02,-6.88521266e-01,-2.42916137e-01]],[[-9.56843615e-01,-1.45372093e-01,-4.77311969e-01],[6.08529225e-02,2.82927126e-01,-3.20014030e-01],[-3.72453183e-01,-3.97349484e-02,-5.64947546e-01]]],[[[2.0223155,-0.24716428,0.80036676],[-0.26020327,0.55817896,-0.22903672],[-0.45687124,0.65661496,0.43049592]],[[-0.29205617,-0.02760541,-0.04927308],[0.6560406,0.49978438,-0.643003],[0.42778662,0.162042,-0.3891108]],[[0.10411102,0.1455449,-1.2363721],[0.27186674,0.04078916,-0.10659016],[-0.25982386,-0.14822266,0.2134685]],[[-0.3234518,-0.10500851,0.38238686],[-0.44529334,-0.02531308,-0.1055088],[-0.00620151,-1.3620509,0.009965]],[[0.48901525,-0.22974046,0.96211654],[-0.3286142,-0.3507686,-0.276306],[0.5078671,0.46794683,0.5195157]],[[0.06888872,0.19517894,0.2443219],[0.32051265,0.12590937,-0.62423927],[-0.4642738,-0.58596647,-0.10572288]],[[0.6366318,-0.2917254,0.32196715],[0.03428958,-0.6646053,0.28105238],[0.01826959,-0.13793272,-0.44792208]],[[0.00401284,0.25680834,-0.3333882],[-1.2273066,-0.30165374,-0.17687018],[-0.82884073,0.44896618,0.63960886]],[[0.13376054,0.4830125,-0.77095956],[0.23618563,-0.03449791,0.12283331],[-0.72779965,-0.57394373,0.18912044]],[[-0.72809565,-0.2798335,-0.03313184],[-0.81762725,-0.328974,0.5035624],[-1.1722063,0.22953232,0.5454271]],[[0.7697859,-0.15641846,-0.1899023],[-0.30030012,-0.2983186,-0.07737081],[0.27366605,0.8999809,0.74106884]],[[0.11774164,-0.30936304,0.05688036],[0.42121923,0.53749484,0.1551897],[0.5932674,0.32869905,-0.4970307]],[[-0.67653686,-0.37526578,0.8511351],[0.3017539,-0.28557932,0.7377196],[0.17641006,0.04841398,0.5270402]],[[-0.84690005,0.11609694,0.3480052],[-0.6171947,-0.11852006,-0.13921241],[0.41288266,0.50879556,-0.16014783]],[[0.3297662,0.69415945,-0.0733588],[-0.3404843,-0.2526398,-0.23039445],[-0.5105115,0.24197803,-0.7736895]],[[0.5348732,-0.4606983,0.6357652],[0.17170662,-0.39676887,-0.25136226],[-0.7228608,-0.13275662,0.48650438]],[[0.09934827,-0.10775193,-0.45482904],[0.137161,0.3275402,-0.1863136],[0.25390923,0.12733826,-0.11488501]],[[-0.39938816,-0.5895073,-0.5607262],[0.2766803,-0.3269353,-0.3978791],[0.18475123,0.59962964,-0.1466898]],[[0.07956782,0.40243727,0.37639943],[-0.6561265,-0.31475732,0.07605302],[0.16394438,-0.08278015,-0.07781267]],[[0.06070643,-0.19226809,-0.48971742],[-0.34210294,-0.07003819,0.32241014],[0.08741736,-0.99058455,0.28012055]],[[-0.30637524,0.29259002,0.41884965],[-0.23084797,0.00905155,0.14191696],[0.11867488,0.10840973,-0.41469476]],[[0.72254133,0.3004026,-0.20351893],[0.46958527,0.5241143,-0.21815856],[0.7048763,-0.31641954,-0.2876511]],[[-0.389923,0.21397652,-0.132214],[-0.17979498,-0.22969493,0.04128449],[0.58806026,0.06129808,0.25773856]],[[-1.2628355,-0.11296842,-0.4742213],[-0.30652314,0.32931578,0.10901653],[0.9790384,0.05935292,-0.12353747]],[[-1.261773,-0.5134571,0.35494143],[0.33467925,-0.28349215,-0.00263463],[-0.07886554,0.19299231,-0.46026984]],[[0.01155696,0.37741137,-0.04949977],[-0.00923123,0.2310308,-0.16880724],[-0.46641257,-0.33269483,0.5291052]],[[0.8838679,0.07859322,0.16749284],[0.18808217,-0.21310414,0.34351924],[-0.13973355,-0.47979692,0.20920171]],[[-0.15340589,0.5176065,0.03756697],[0.38138506,0.095686,-0.14636274],[-0.24877001,-0.31060407,-1.0517185]],[[0.26630405,0.11785071,0.41323325],[-0.7834035,-0.7883311,0.14096688],[-0.04841463,-0.48338822,-0.09014741]],[[-0.57132536,-1.0044272,-0.6369138],[0.47510645,0.28070855,0.13141114],[0.6904585,0.21819589,-0.5195357]],[[-0.25277463,0.03264854,-0.33789983],[0.5742311,-0.06466119,0.06584501],[-0.3643299,-0.42561632,0.40554965]],[[0.3362906,0.17669801,-0.4953339],[1.3725512,-0.6187327,-0.29349855],[-0.7650244,0.57122815,0.33166042]],[[0.12596636,-0.12509038,-0.00325394],[-0.31891173,-0.03430242,0.02177319],[-0.87307465,0.34660685,-0.02304649]],[[0.02112627,0.01845151,-0.08174037],[-0.2977346,-0.4193264,0.39571422],[0.28295133,-0.07913991,0.14031549]],[[-0.05543194,0.40476853,0.598332],[0.18358722,0.046371,-0.3014518],[0.9159238,0.185902,0.34296066]],[[-0.8340438,0.35587725,0.01323504],[-0.5537801,0.07381719,0.6894662],[0.05648779,-0.2886207,0.77255845]],[[-0.24405134,0.5675868,-0.22259001],[0.19201323,0.36362284,0.3679114],[0.57774997,-0.5251056,0.4112367]],[[0.63755167,-0.5386597,0.2522344],[0.24874468,-0.20264967,-0.19370188],[-0.91398793,0.07740954,-0.3475682]],[[1.2297902,0.66477644,-0.472094],[0.1035334,0.14126016,0.4922072],[-0.12705892,-0.36335823,0.92898285]],[[-0.78115565,0.74133,0.37024418],[-0.7727187,-0.45609638,-0.41169116],[-0.5223131,0.13351494,0.70441455]]],[[[1.96074843e+00,2.15455160e-01,1.14945229e-02],[1.06330645e+00,2.96125188e-02,7.13718355e-01],[1.91842213e-01,-2.64644355e-01,-3.01252212e-02]],[[-1.22013855e+00,4.21366900e-01,-2.98366398e-01],[-1.63704038e-01,4.14625764e-01,2.90451258e-01],[-3.95759828e-02,-1.70104682e-01,1.67928692e-02]],[[-3.23192850e-02,-2.89232910e-01,-2.73220465e-02],[-1.18761100e-01,2.46357635e-01,-6.05607554e-02],[-4.01783675e-01,4.02756333e-01,-5.00946522e-01]],[[3.04410517e-01,9.56641361e-02,-3.85597944e-01],[-8.48155141e-01,-2.61829972e-01,6.48948550e-02],[-1.29522666e-01,1.13853917e-01,3.25186282e-01]],[[6.00098729e-01,-4.45523411e-01,7.05881566e-02],[5.67799866e-01,-1.36993551e+00,-3.69607270e-01],[3.42946827e-01,-1.07154950e-01,4.49575365e-01]],[[-6.55045137e-02,-4.30906028e-01,2.46566519e-01],[4.86756163e-03,8.06662142e-01,-7.51194894e-01],[-2.23558396e-01,9.23399806e-01,1.35760665e-01]],[[-1.54477134e-01,-3.20331037e-01,4.83890235e-01],[4.59109485e-01,7.46593595e-01,1.57149985e-01],[1.48659483e-01,9.88161564e-02,1.00327706e+00]],[[1.59627929e-01,-1.08031869e-01,-5.13876081e-02],[1.57093583e-03,4.17135209e-02,2.11063474e-02],[-5.00813842e-01,-2.72679180e-01,-2.30194718e-01]],[[-3.93115431e-01,-3.76965106e-01,-3.79117370e-01],[-3.22565287e-01,-1.31721735e-01,-6.59101784e-01],[4.42191929e-01,6.31590486e-01,-1.35643734e-02]],[[-4.86722529e-01,-2.55976796e-01,-7.77668238e-01],[1.44970641e-01,-6.39499366e-01,-2.41679505e-01],[-1.46557152e-01,-1.01034909e-01,4.19278979e-01]],[[-9.03096557e-01,3.96056712e-01,2.59879887e-01],[1.06224167e+00,-1.38445526e-01,7.78345987e-02],[8.02613497e-01,1.44374505e-01,2.21448421e-01]],[[7.03925014e-01,2.29227357e-02,5.82835853e-01],[-6.89158291e-02,1.57059595e-01,1.31249800e-01],[2.79693156e-01,-3.24375868e-01,-4.16635066e-01]],[[2.47612283e-01,-3.62428054e-02,-1.85876235e-01],[-3.80218215e-02,-2.28627250e-01,-9.56864357e-01],[4.51490760e-01,-5.28780282e-01,-5.91598749e-01]],[[-1.54321129e-02,9.14077535e-02,-2.06127465e-01],[-3.16296071e-01,-1.15461610e-01,-5.48013866e-01],[3.37046295e-01,8.03828835e-02,9.47837055e-01]],[[1.74290404e-01,6.35118783e-02,-3.59407097e-01],[6.53991103e-01,5.12450561e-02,-5.01107648e-02],[-9.40827250e-01,5.13517022e-01,-9.82752383e-01]],[[-7.30713367e-01,-5.88447265e-02,-1.01630010e-01],[3.76739711e-01,-4.02020514e-01,6.22017562e-01],[3.02574605e-01,-8.19550753e-01,6.13788962e-01]],[[5.61612695e-02,5.43694258e-01,1.09350540e-01],[1.98713750e-01,-6.20054901e-01,1.87693805e-01],[1.58138275e-01,5.24906330e-02,5.94567299e-01]],[[-1.68028578e-01,1.63465708e-01,2.76862253e-02],[-5.94324887e-01,3.81467849e-01,1.38721794e-01],[3.53558175e-02,3.84149283e-01,9.98798311e-01]],[[-5.99032581e-01,-8.88023794e-01,-1.90768704e-01],[2.22080544e-01,-3.60020369e-01,-2.72380859e-01],[-3.43499631e-01,1.09688783e+00,4.49276298e-01]],[[-5.55679202e-01,2.47374743e-01,-2.65825331e-01],[-1.48628831e-01,-3.46729815e-01,-8.64563048e-01],[1.00607969e-01,2.35549748e-01,-2.52795033e-02]],[[-7.65701711e-01,7.27982819e-02,2.56256998e-01],[8.28279778e-02,2.76500106e-01,5.76413393e-01],[-1.19986214e-01,2.36362517e-01,4.04141992e-01]],[[1.15340161e+00,1.96672112e-01,3.11300278e-01],[8.95164251e-01,3.09317876e-02,-5.37456930e-01],[-3.18469524e-01,-4.98478979e-01,2.74296105e-01]],[[2.30021223e-01,1.95948660e-01,1.71181858e-02],[1.01756066e-01,-5.49070299e-01,-3.84650938e-02],[4.43597913e-01,-2.39369914e-01,1.15970857e-01]],[[3.59052896e-01,1.63015246e-01,-3.95517498e-01],[6.81131721e-01,-3.68180394e-01,2.40324676e-01],[-6.99808300e-02,4.25342023e-01,-2.22847119e-01]],[[-1.98052198e-01,-1.83992162e-01,-5.03666520e-01],[-1.59611613e-01,-1.83481634e-01,7.47818798e-02],[4.83787693e-02,3.54267359e-01,2.46169478e-01]],[[-4.04237956e-01,1.85395405e-01,1.12568028e-01],[2.94968486e-01,5.33113897e-01,-2.65672147e-01],[4.02720153e-01,2.83561081e-01,1.42367363e+00]],[[8.95706892e-01,-1.07234299e-01,-5.33381939e-01],[7.65492678e-01,-2.18151957e-01,-7.36002505e-01],[-3.81960511e-01,4.98460472e-01,2.10383534e-01]],[[-5.51949322e-01,3.58215988e-01,-5.92868567e-01],[4.45323080e-01,-4.05983239e-01,1.65387735e-01],[-1.17877573e-01,-4.53783929e-01,9.51856911e-01]],[[5.85923731e-01,2.74731964e-01,6.71801746e-01],[-2.28017420e-01,-3.07539225e-01,2.71361560e-01],[4.19731319e-01,-2.74083614e-01,2.13588879e-01]],[[-4.76094559e-02,-8.01487565e-01,2.49359846e-01],[-2.73118705e-01,1.01069123e-01,2.06184387e-01],[4.24357643e-03,-2.01706693e-01,-1.11622758e-01]],[[2.60838270e-01,4.96514469e-01,3.86593521e-01],[5.40531129e-02,-1.76679432e-01,8.19841623e-02],[-2.31442973e-01,-4.39206600e-01,-2.66305469e-02]],[[3.97300541e-01,-1.31292967e-02,-9.84434858e-02],[5.19497275e-01,2.31570318e-01,-3.69144976e-01],[-4.47392881e-01,6.26572192e-01,-7.07009315e-01]],[[-8.63110006e-01,5.71559556e-02,-4.49612737e-01],[-4.68322225e-02,8.72362316e-01,-2.19167024e-01],[6.41722500e-01,2.30784371e-01,-4.97550480e-02]],[[9.58495811e-02,5.99973321e-01,5.15633166e-01],[5.14796376e-01,-5.03053904e-01,1.16377223e+00],[5.08512676e-01,2.69428998e-01,1.44635677e-01]],[[8.17678332e-01,-4.24971767e-02,8.38284075e-01],[-1.34599254e-01,8.14158246e-02,-1.71245877e-02],[4.09327060e-01,-3.29688042e-01,9.13404375e-02]],[[4.94329602e-01,-6.00975037e-01,-6.04634620e-02],[7.17869028e-02,-3.33897397e-02,-1.71294481e-01],[2.51077950e-01,-5.32428287e-02,1.31795958e-01]],[[4.84809339e-01,3.95410843e-02,-4.87300813e-01],[-2.24804625e-01,1.29693404e-01,-1.11819454e-01],[7.38112271e-01,3.23457032e-01,-2.31561646e-01]],[[9.37457800e-01,-2.02799782e-01,-5.74761033e-01],[1.66821126e-02,-3.24071258e-01,-1.79220110e-01],[-5.60218871e-01,-6.95578277e-01,4.14413780e-01]],[[3.88583988e-01,-2.92285066e-02,-1.50479317e-01],[3.36388908e-02,-4.09453213e-01,-1.02349585e-02],[3.84547710e-01,2.77324021e-01,-5.75052619e-01]],[[-1.26266897e+00,8.72750394e-03,-1.72008380e-01],[3.62320900e-01,2.35510111e-01,-3.46166700e-01],[3.09840709e-01,-6.86477870e-02,4.90397185e-01]]],[[[9.61019218e-01,-3.80492449e-01,-2.08200812e-01],[3.20074260e-01,2.70405766e-02,5.17390609e-01],[-2.48736158e-01,3.02230746e-01,-2.61901259e-01]],[[-2.44090557e-01,3.58102858e-01,4.88885254e-01],[1.72462210e-01,3.94047290e-01,2.34016702e-01],[4.09841031e-01,-2.68115401e-01,-3.02247200e-02]],[[1.89631343e-01,3.72093350e-01,1.63744450e-01],[-2.97741652e-01,1.11997016e-02,2.70249963e-01],[4.73224282e-01,6.72308087e-01,7.87006468e-02]],[[2.13355973e-01,4.76858974e-01,1.37613043e-01],[1.78175885e-02,2.94790953e-01,-1.31388515e-01],[7.31999576e-02,8.80777240e-01,-5.25250845e-02]],[[2.85546839e-01,-1.41792163e-01,2.35211998e-02],[8.29310775e-01,3.79231453e-01,2.11262584e-01],[2.50412554e-01,6.36194289e-01,4.96563792e-01]],[[4.38718051e-01,5.93524873e-01,2.61721492e-01],[5.17800391e-01,-4.80811685e-01,5.33207715e-01],[-5.28168321e-01,6.72685653e-02,6.63870394e-01]],[[1.71354547e-01,1.42779097e-01,1.65701896e-01],[-1.26865551e-01,1.00164020e+00,7.17769325e-01],[2.58467477e-02,-6.91314757e-01,4.08006161e-02]],[[2.44712040e-01,-2.78670639e-01,2.43825689e-01],[4.36712712e-01,2.95283914e-01,3.99993479e-01],[-1.49552882e-01,2.43581951e-01,6.23576865e-02]],[[6.37605608e-01,3.99674848e-02,2.25342199e-01],[8.66757274e-01,-6.44322932e-01,4.02505435e-02],[-5.50927937e-01,3.68219554e-01,3.65762264e-01]],[[3.40774149e-01,1.24339974e-02,-3.65662538e-02],[9.41642821e-01,9.48376119e-01,8.19970250e-01],[9.62720096e-01,6.55598998e-01,1.90918490e-01]],[[-6.00802600e-01,-1.56881839e-01,-2.30571151e-01],[-7.54817724e-01,-3.66296053e-01,2.86557794e-01],[6.72977744e-03,1.03862606e-01,6.97871804e-01]],[[-3.78357083e-01,2.18349770e-02,8.04794282e-02],[7.37731084e-02,-4.22951430e-02,3.07290524e-01],[2.78356075e-01,2.67311245e-01,3.10402781e-01]],[[-9.37608480e-02,-6.24521255e-01,4.41622227e-01],[-5.66339016e-01,-3.83494824e-01,4.39902455e-01],[8.00098062e-01,-2.54816562e-01,-8.62497762e-02]],[[-4.83368337e-01,2.83390805e-02,3.85879457e-01],[-1.99671392e-03,3.50359499e-01,-3.86855416e-02],[6.07815027e-01,-2.03713104e-01,1.40494645e-01]],[[2.97235996e-01,-2.02394612e-02,-4.50836957e-01],[-2.90647626e-01,-1.86565265e-01,-1.04986362e-01],[-7.98090994e-01,7.94095397e-01,-2.58132845e-01]],[[2.26290524e-01,-3.53828818e-01,-1.39329463e-01],[7.34394550e-01,2.31839642e-01,-9.73781645e-01],[2.35032812e-01,1.09576494e-01,3.31562728e-01]],[[-6.57843128e-02,-2.67935514e-01,-3.12851042e-01],[6.75091565e-01,5.09072602e-01,5.33907175e-01],[-5.18533468e-01,7.77759850e-01,-3.75108927e-01]],[[2.45363787e-01,-9.10693780e-02,-2.88942277e-01],[9.29823577e-01,3.46220046e-01,5.49016818e-02],[-6.99879408e-01,-3.18055242e-01,-1.37033209e-01]],[[-3.94116521e-01,4.42282319e-01,4.09998953e-01],[5.63145876e-01,2.33965710e-01,-2.07435098e-02],[-2.33796448e-01,3.40278417e-01,6.55549407e-01]],[[1.77858487e-01,4.49917883e-01,-6.95959806e-01],[1.54637262e-01,-1.11740246e-01,3.20980966e-01],[-5.15909120e-02,7.17402473e-02,3.83433759e-01]],[[5.71372807e-02,1.02565277e+00,2.74062037e-01],[-1.51465476e-01,-2.27810025e-01,5.61325133e-01],[4.37693208e-01,-6.37224019e-01,-3.78683239e-01]],[[-9.44213152e-01,5.92599869e-01,-3.37875545e-01],[-1.15668520e-01,3.50530088e-01,-3.96295547e-01],[-4.08676714e-02,-1.38109088e-01,-3.03483814e-01]],[[1.37320921e-01,-1.42839353e-03,1.55651224e+00],[-2.32584283e-01,2.16031075e-01,5.27417600e-01],[3.96211326e-01,1.61160007e-01,-1.02456301e-01]],[[-3.26572597e-01,-1.71334997e-01,3.36843133e-01],[-2.23373305e-02,6.96884453e-01,-4.61205930e-01],[1.62021804e+00,6.20909035e-01,-9.56332311e-02]],[[-1.42210943e-03,9.49763507e-02,-1.18882045e-01],[1.93566516e-01,-2.19927594e-01,-7.63472468e-02],[-4.47811000e-02,-7.13974893e-01,3.43474699e-03]],[[1.14108711e-01,9.52703059e-02,6.61443591e-01],[1.95112512e-01,7.87605643e-01,-2.14333177e-01],[-5.15889943e-01,1.27873033e-01,3.86110604e-01]],[[3.71049494e-01,-4.50382769e-01,-4.23284352e-01],[7.11175427e-03,8.43777955e-02,3.35208625e-01],[-2.84843653e-01,-1.22941323e-01,2.46519163e-01]],[[7.25296557e-01,2.34507561e-01,-4.48290646e-01],[-1.18422620e-01,2.52002746e-01,-3.24721485e-02],[-1.11409359e-01,-3.25165570e-01,9.71518874e-01]],[[3.61485109e-02,2.02172667e-01,-9.46364045e-01],[6.31562471e-01,3.06253701e-01,3.77264917e-01],[-4.11640167e-01,-2.97150493e-01,-1.55689240e-01]],[[4.88150984e-01,-1.72563285e-01,4.65361536e-01],[4.48163122e-01,5.02355099e-01,2.04245001e-02],[8.78526792e-02,-1.47720188e-01,4.51407552e-01]],[[-1.67549774e-01,-4.29811001e-01,-5.32926284e-02],[-1.91921126e-02,-2.28050679e-01,8.30862969e-02],[-3.23468596e-01,-1.84650533e-02,-3.58286053e-01]],[[8.78337443e-01,1.39386684e-01,-1.08157229e-02],[1.02324098e-01,-1.93257853e-01,-4.65670466e-01],[-1.06877975e-01,-1.27798033e+00,1.10611880e+00]],[[7.94490695e-01,2.07432613e-01,-4.66130376e-01],[7.36952484e-01,-7.75952756e-01,3.14149112e-01],[-4.36134696e-01,-5.64639509e-01,-6.45906329e-01]],[[-1.05156884e-01,2.97906455e-02,-4.89121884e-01],[-4.01748598e-01,-2.22205713e-01,-8.30510408e-02],[-1.29841030e+00,3.03886145e-01,-4.62270260e-01]],[[6.82897508e-01,-1.65517911e-01,-3.06114286e-01],[9.34535265e-01,-2.19208539e-01,-2.88821101e-01],[5.13890147e-01,-2.52203673e-01,2.05958322e-01]],[[-1.26388267e-01,9.62058067e-01,-8.64635631e-02],[-1.55207515e-01,6.20886147e-01,3.95267695e-01],[3.77607644e-01,-5.93064427e-01,-8.17238390e-01]],[[8.84409398e-02,-6.64218366e-01,-4.71550114e-02],[1.75443724e-01,7.18560591e-02,3.35275918e-01],[9.30550098e-02,-3.87452245e-02,-2.36426279e-01]],[[-5.84688008e-01,-8.04539382e-01,-1.30877531e+00],[-6.28521889e-02,6.44682348e-01,-3.04310769e-02],[6.26257122e-01,3.41905981e-01,5.33276796e-01]],[[2.93812960e-01,-2.41740420e-02,2.25442559e-01],[-7.91839659e-02,-2.26443022e-01,-6.77004457e-01],[5.48326790e-01,-2.01176107e-01,2.05397516e-01]],[[-9.50527012e-01,2.96174586e-02,-5.52908778e-01],[1.07668497e-01,-1.80139616e-01,-5.08244634e-01],[2.96456039e-01,-1.02288112e-01,-7.77101159e-01]]],[[[0.06150795,-0.05732618,-0.42220515],[-1.109886,-0.5876984,0.30337188],[-0.5362435,-0.22144184,-0.25424555]],[[0.12165074,0.33282188,0.6431469],[0.43931043,-0.7091524,0.54250395],[-0.09683045,-0.30694816,-0.08861645]],[[0.08140126,-0.24207322,0.51606435],[-0.5816567,-0.53378,-0.06627385],[0.00569939,-0.20063055,-0.30302238]],[[-0.70845234,0.14667708,0.00582111],[0.11807809,-0.12964493,0.33368912],[-0.37075776,0.0719481,1.0468601]],[[-0.28515467,-0.349122,0.13749109],[-0.24160174,-0.5106638,0.13931835],[-0.7027428,0.02560525,-0.18735705]],[[-0.2816329,-0.2257651,0.19800943],[0.21108735,0.07307845,0.3179216],[0.10985673,0.12332012,-0.21124995]],[[0.61681414,-0.18278368,-0.19911076],[-0.40060392,0.63241434,-0.13332267],[1.3911091,0.4288118,-0.23968887]],[[0.6396901,-0.37240863,-0.05930971],[0.44583786,0.38599983,0.52726036],[1.0656018,0.3575722,0.47710922]],[[-0.19776382,-0.43244636,0.00662331],[0.6047807,0.60558283,-0.4015031],[0.15868503,-0.09626909,-0.0262507]],[[1.0063999,-0.0739508,-0.0655231],[0.41065925,0.31416786,0.67112654],[0.5727768,1.256742,0.77865267]],[[0.23824699,-0.04361862,0.38941056],[0.01680854,-0.16780508,-0.00496958],[-0.35808867,0.49843282,-0.5922678]],[[-0.7214823,-0.04176902,0.06786483],[-0.31529215,-0.3579475,-0.58771294],[0.3672464,-1.1920422,-0.15579043]],[[-0.9984516,-0.3560795,-0.13575226],[0.977807,0.628586,0.03938429],[-1.1059853,0.54606515,-0.41035885]],[[-0.39520812,-0.09245566,-0.47206244],[0.343728,0.1680041,1.0679871],[0.10772365,1.1091213,0.08624399]],[[-0.13934486,0.22890057,0.14267746],[-0.35394645,-0.01861164,-0.3101369],[0.4989473,-0.0547883,0.66280055]],[[-0.35574332,0.08742489,0.34301484],[0.29749206,0.6564419,0.17313112],[0.3410593,0.26173022,0.62715125]],[[0.28927925,-0.11471555,-0.88184303],[-0.12567718,-0.4194159,0.29849392],[-0.00147566,-0.06441122,0.5436951]],[[-0.9451175,0.21708588,0.1244552],[0.19351184,0.23988876,-0.09325269],[-0.05589949,-0.29783005,0.795988]],[[-0.3759707,0.18205728,0.2106725],[-0.31952682,0.98794717,-0.4634611],[0.30300122,0.22917053,-1.4632888]],[[0.48539576,-0.08809967,-0.12119933],[-0.19563161,-0.35519025,0.04746326],[-0.51046133,0.08080044,-0.12138601]],[[-0.24222891,0.12202805,0.29462612],[-0.25765762,0.33056223,-0.12680015],[-0.40730217,0.06028865,-0.04889114]],[[-0.65343654,-0.15043546,-0.35338506],[0.5289492,-0.07658358,0.12260229],[0.3196025,0.1394909,-0.49925914]],[[-0.09360889,0.21290976,0.11099306],[-0.02313117,-0.05108413,-0.07184799],[-0.4321147,-0.19852778,-0.36919728]],[[-0.07614541,0.10248221,0.53623533],[0.23794423,-0.45920992,-0.25695172],[-0.3120125,0.07008417,-0.04916304]],[[0.02045776,-0.65142083,0.25495383],[-0.20425193,0.23832822,-0.24656765],[0.5580286,0.13724865,0.2043657]],[[-0.3584601,0.4662957,-0.7727582],[0.524658,-0.14480984,-0.19933534],[0.7174183,1.257136,-0.16073675]],[[-0.71897805,-0.7067043,-0.37944183],[0.4518298,0.8907034,0.41413647],[0.22856525,0.07293828,-0.94778746]],[[-0.03932703,0.43741196,-0.23615667],[-0.0746134,0.05051655,0.3801951],[0.8078815,0.5308433,-0.21451642]],[[-0.19345611,-0.18982871,0.5306418],[0.46067363,0.06991634,-0.07591482],[-0.17188731,-0.35643223,0.09419508]],[[-0.59625226,0.12033536,0.5864207],[0.9062559,0.22506748,-0.1559272],[-0.16387165,0.04898478,-0.20112303]],[[-0.2564793,0.00863923,-0.09837293],[0.03233533,-0.6395289,0.16101275],[-0.57785887,-0.2576909,0.31045732]],[[-0.23418732,-0.03094551,-0.09517289],[-0.23396628,-0.49377167,-0.17602028],[0.19802363,1.1052852,0.22692348]],[[-0.6138243,-0.7483632,0.3339506],[0.77079064,-0.5274503,-0.17993452],[0.42740214,-0.47492817,-0.1056995]],[[0.2875258,-0.5703439,0.6142951],[0.58142114,0.25067258,0.18824741],[0.00949604,-0.533423,-0.7436491]],[[0.08082736,0.29189143,0.23992705],[0.26890048,0.05186174,0.66476387],[-0.01852921,0.1228032,0.5087624]],[[0.177401,0.10465129,1.0112104],[-0.35784099,0.39552554,-1.3876983],[0.07897502,-0.36522794,-0.7458899]],[[-0.29873064,0.4319461,0.76068044],[0.25789103,0.15639398,0.08900941],[0.79238653,-0.71505845,0.03826708]],[[0.77128524,0.45864567,0.0145671],[0.01796244,0.0358905,-0.4178037],[0.19669844,0.6707482,0.22213241]],[[0.47814095,0.3904805,-0.56162304],[0.41375458,0.1793742,0.7447832],[0.26043215,0.19979568,-0.2943999]],[[0.46988636,-0.68465585,0.49041376],[-0.3163902,0.24953327,0.12426652],[0.43281054,-0.19655296,0.06222211]]],[[[8.91076982e-01,2.29432181e-01,-4.75115210e-01],[-5.23975492e-02,3.53681892e-01,-2.22333625e-01],[-6.44556507e-02,-6.98720396e-04,-3.71393740e-01]],[[9.35176667e-03,-3.84568274e-01,-3.74924801e-02],[-7.73968473e-02,-2.60701239e-01,-2.01527715e-01],[-1.35524288e-01,1.59561709e-01,2.99327374e-01]],[[1.82082146e-01,2.91034877e-01,4.36205029e-01],[1.73330262e-01,-1.57176465e-01,1.11539841e+00],[-1.59894720e-01,-6.86967254e-01,4.01318878e-01]],[[-1.88546240e-01,-8.67939815e-02,1.13237910e-01],[3.51238370e-01,-8.51029336e-01,1.48908183e-01],[1.75536722e-02,-3.20171624e-01,-7.87871003e-01]],[[9.11570527e-03,-2.50018127e-02,3.58452618e-01],[-1.45967782e-01,4.94960517e-01,-4.92652148e-01],[-4.41183120e-01,-5.18324316e-01,-4.49872077e-01]],[[4.74045068e-01,3.77937883e-01,1.15437075e-01],[-2.06314802e-01,1.09242581e-01,1.90256312e-01],[-6.95653200e-01,-2.51905024e-01,-3.91073793e-01]],[[2.15205297e-01,-8.49941969e-02,8.81044865e-01],[3.42101842e-01,-3.04726928e-01,-6.37560785e-01],[9.58049223e-02,-4.58644688e-01,1.47673941e+00]],[[-4.49055195e-01,-1.16010152e-01,6.89154983e-01],[-6.75411105e-01,2.84409612e-01,-2.25053266e-01],[5.54062203e-02,3.55208516e-01,-4.23357874e-01]],[[-1.09194851e+00,3.03679287e-01,-1.07688129e-01],[-4.58985686e-01,-3.48024428e-01,-7.45028913e-01],[5.35874605e-01,5.26754797e-01,4.50173765e-02]],[[-7.38491356e-01,-3.17998201e-01,-1.16191581e-01],[-9.36568260e-01,-3.66704941e-01,-7.76678979e-01],[-4.28916991e-01,1.66284993e-01,-2.50656486e-01]],[[-2.80445576e-01,6.14331961e-02,3.54215533e-01],[4.13571656e-01,4.88650590e-01,-2.96584785e-01],[-5.01177847e-01,-2.13824555e-01,-7.51863360e-01]],[[2.15121537e-01,1.33994743e-02,6.17478430e-01],[-1.86706573e-01,2.84912854e-01,1.34476259e-01],[-2.93391384e-02,-2.94327259e-01,1.08346617e+00]],[[-3.86695415e-02,4.64228749e-01,-2.44656354e-01],[1.25545964e-01,2.91294456e-01,-5.79708874e-01],[-6.32142186e-01,1.73964918e-01,-8.40025663e-01]],[[-3.63793224e-01,-1.61398605e-01,1.91648286e-02],[-3.32473636e-01,-1.24077266e-02,-2.35401429e-02],[-3.40019986e-02,7.39427004e-03,-3.05592567e-01]],[[4.46240939e-02,5.14645457e-01,-8.63108039e-01],[-8.06779116e-02,-1.06545761e-01,3.44542041e-02],[-1.46352991e-01,3.19559962e-01,-1.85137168e-01]],[[2.21860543e-01,5.58031559e-01,-7.27017105e-01],[3.53356898e-01,4.70866621e-01,-2.23848552e-01],[1.65099427e-01,-2.91219708e-02,-5.70227206e-01]],[[-2.01715991e-01,3.01088363e-01,6.43901467e-01],[-7.24551201e-01,-3.83533925e-01,7.86584854e-01],[-7.05162585e-02,-4.98251677e-01,-6.68530524e-01]],[[2.14149699e-01,8.29548776e-01,8.68633747e-01],[2.87271380e-01,3.17697704e-01,-5.32452166e-01],[-1.85334891e-01,6.64766550e-01,5.92686534e-01]],[[1.23474278e-01,-6.04400277e-01,-3.48682135e-01],[-5.93488991e-01,-5.15701532e-01,-6.12393655e-02],[4.87843841e-01,-6.36883974e-01,3.18231657e-02]],[[-2.91998893e-01,-1.61033154e-01,7.34047666e-02],[7.51428783e-01,-8.30613598e-02,1.42481208e-01],[-5.32512665e-01,9.34046328e-01,-3.57703745e-01]],[[3.35716493e-02,-5.43293774e-01,-7.40748048e-01],[-3.70901525e-01,4.07863408e-01,-3.29820246e-01],[-3.94553661e-01,1.18258454e-01,-4.51858461e-01]],[[4.17827517e-01,4.18691337e-01,-1.03508033e-01],[1.08351600e+00,2.22410038e-01,-3.49191666e-01],[1.40002504e-01,4.65437144e-01,-2.63059288e-01]],[[2.23768763e-02,-1.53016001e-01,-4.59082097e-01],[-4.84171987e-01,-1.04244024e-01,-7.53305435e-01],[-2.07984135e-01,-6.18162096e-01,-2.02713281e-01]],[[2.76271671e-01,3.12444091e-01,9.66852009e-01],[5.81598580e-01,6.49987578e-01,-3.69866163e-01],[-5.20210154e-02,-1.12521827e-01,7.69641772e-02]],[[-4.15374935e-01,3.60807240e-01,-9.74963784e-01],[4.18720812e-01,-4.58048612e-01,-5.91724515e-02],[1.58847928e-01,1.90209940e-01,-7.71260679e-01]],[[-4.22631502e-01,4.53095973e-01,3.35974038e-01],[-5.16398430e-01,-4.11433727e-02,-3.46320480e-01],[3.16849202e-01,2.75434852e-01,6.33013546e-01]],[[6.76151454e-01,-3.16301018e-01,-4.57905978e-01],[3.99060905e-01,-1.04049730e+00,-4.68624890e-01],[-6.13018274e-01,3.54034066e-01,1.92718849e-01]],[[-3.62549394e-01,4.12025124e-01,-2.61888236e-01],[-4.67365198e-02,-9.24234465e-02,-3.92446011e-01],[5.77590525e-01,-3.12373281e-01,2.64422428e-02]],[[3.56555684e-03,1.18999757e-01,-4.68785137e-01],[2.23428786e-01,-1.72124337e-03,-6.34629250e-01],[3.10451537e-01,1.78574353e-01,8.22646260e-01]],[[-9.61457014e-01,-3.68101656e-01,6.44912198e-02],[-1.20138988e-01,-7.67600417e-01,2.95570940e-01],[1.32365093e-01,1.39548513e-03,3.71839285e-01]],[[-7.17280626e-01,-1.49235189e-01,-1.47113812e+00],[3.28197069e-02,1.46557212e-01,-3.54224667e-02],[-6.42843425e-01,1.27760274e-02,7.56880119e-02]],[[-9.14008543e-02,3.65916312e-01,1.94129869e-01],[-6.00434601e-01,-7.44929686e-02,-3.76569070e-02],[9.57499146e-01,1.06856726e-01,-3.11477959e-01]],[[-8.30366135e-01,-4.84331995e-01,-9.99503195e-01],[1.06000513e-01,-5.81771314e-01,-2.73585349e-01],[-7.75624514e-01,4.41433102e-01,2.29063213e-01]],[[1.54765591e-01,6.29780352e-01,1.69011280e-01],[5.23154080e-01,4.14141178e-01,1.06233776e-01],[-7.54523277e-01,3.26439351e-01,4.13196385e-01]],[[-3.26603234e-01,7.86854178e-02,7.96354413e-02],[1.69319347e-01,8.06470215e-02,6.37404770e-02],[-1.55860260e-01,-1.85461745e-01,2.45339677e-01]],[[3.58633348e-03,1.53786004e-01,-5.84323347e-01],[-8.24359536e-01,2.11398870e-01,-1.92071259e-01],[5.15950859e-01,-9.37140346e-01,-1.56799629e-01]],[[-4.05786932e-01,-3.92735392e-01,1.16580844e+00],[-2.34870967e-02,6.08488202e-01,-5.15374243e-01],[1.06191456e-01,1.18505685e-02,-1.51631996e-01]],[[-1.85695495e-02,6.23577237e-01,1.09069443e+00],[-2.31910974e-01,4.82286513e-01,2.78655052e-01],[-6.96588635e-01,-6.23448670e-01,3.52041781e-01]],[[8.98467302e-01,1.87656865e-01,8.52929771e-01],[6.30977094e-01,2.53824681e-01,4.83180463e-01],[7.76311532e-02,-8.12484026e-02,2.79858381e-01]],[[2.32373714e-01,-4.94087994e-01,-4.12426800e-01],[-3.87888998e-01,-6.34368420e-01,2.04069942e-01],[-5.59220016e-01,-3.48147094e-01,-1.74843609e-01]]],[[[-1.58533797e-01,2.97022760e-01,8.41766059e-01],[-1.62440658e-01,1.39479741e-01,-3.89609188e-01],[-8.26885626e-02,-2.88718969e-01,3.53255033e-01]],[[-9.54631746e-01,-3.67496729e-01,-4.09087270e-01],[-5.92791378e-01,-3.83056730e-01,7.28969350e-02],[-6.67312443e-02,-1.49917901e-01,6.27393648e-02]],[[-6.83086753e-01,7.95312896e-02,3.58102679e-01],[-7.18357384e-01,-2.68817604e-01,-3.59764308e-01],[2.70186007e-01,-3.32272142e-01,5.11869133e-01]],[[-6.55872285e-01,4.92268413e-01,-1.42976502e-02],[1.94517493e-01,4.97566104e-01,2.90876459e-02],[-6.93969280e-02,4.43852365e-01,4.16367576e-02]],[[-5.02376482e-02,8.74039065e-03,3.24964732e-01],[3.64474833e-01,-3.17038178e-01,-7.71926939e-01],[-3.34337413e-01,6.27394319e-01,3.30776274e-01]],[[-3.67386848e-01,1.55353263e-01,1.76991433e-01],[-4.80997115e-01,3.22955281e-01,-3.10913563e-01],[1.07656097e+00,-3.88809741e-01,-5.17751098e-01]],[[2.16611385e-01,-3.63751173e-01,-1.15789734e-01],[-2.99533695e-01,4.31299925e-01,-1.12944305e-01],[-3.10069323e-01,-5.52366823e-02,1.78387433e-01]],[[-3.69885052e-03,2.24941328e-01,6.13873065e-01],[9.54772830e-02,-2.67530560e-01,-3.10373664e-01],[2.53502846e-01,3.29730809e-01,-3.01741548e-02]],[[-8.89326930e-02,-8.68607089e-02,-7.35542417e-01],[2.06237480e-01,-5.38912594e-01,2.55100608e-01],[1.53023556e-01,-7.35458076e-01,-4.49532680e-02]],[[-3.21903914e-01,9.85218063e-02,3.78969461e-01],[-6.10564053e-02,5.73721409e-01,-5.34160376e-01],[7.24965811e-01,-2.31531307e-01,-4.60391715e-02]],[[-7.43852258e-01,-6.58462718e-02,1.53139323e-01],[1.04891375e-01,-2.62739152e-01,-5.14607847e-01],[2.67670065e-01,3.47295046e-01,3.95125985e-01]],[[2.15329394e-01,-1.12741299e-01,2.83887446e-01],[-8.63434374e-01,6.03510499e-01,6.91947997e-01],[-9.70579028e-01,-5.22636592e-01,-7.00421035e-01]],[[1.45552322e-01,-4.29040849e-01,4.26105350e-01],[-4.78891701e-01,1.85334861e-01,-4.79370564e-01],[-6.67767286e-01,-1.30492657e-01,-5.03595293e-01]],[[-5.89868248e-01,-7.76164591e-01,7.17455208e-01],[-1.60200998e-01,-4.22950029e-01,3.01455945e-01],[1.17088743e-01,-5.49800515e-01,-6.29345119e-01]],[[6.88250840e-01,-1.64721951e-01,1.90900579e-01],[-8.08935046e-01,-9.90089178e-02,-4.55667198e-01],[-3.70860606e-01,2.68212706e-01,-2.18011066e-01]],[[-6.49395168e-01,2.60245949e-01,6.76128045e-02],[1.60874352e-02,-2.51477957e-01,1.87487245e-01],[6.99070608e-03,7.38083422e-01,3.76554936e-01]],[[-5.31206787e-01,6.93262875e-01,6.34404868e-02],[2.20152080e-01,-1.13305554e-01,-1.86506033e-01],[2.98796296e-01,-1.48317352e-01,-1.57260776e+00]],[[-4.28570211e-01,4.74935740e-01,-3.35272789e-01],[-5.23740888e-01,5.58347777e-02,1.03486948e-01],[-1.16796844e-01,4.87257570e-01,-5.71422040e-01]],[[-3.70026268e-02,-2.82528073e-01,-7.53333449e-01],[4.11645174e-01,-3.87586594e-01,5.06061554e-01],[-3.73237252e-01,2.48640999e-01,-7.51541033e-02]],[[-2.35242713e-02,2.49415815e-01,-1.04443647e-01],[1.53991088e-01,-7.69393593e-02,3.71377217e-03],[-2.72712439e-01,1.19262680e-01,2.01179534e-01]],[[1.58525690e-01,-3.13723385e-01,7.62023637e-03],[-9.81571749e-02,-5.56138679e-02,3.55557889e-01],[1.65151611e-01,1.64979577e-01,6.45441785e-02]],[[6.30859375e-01,1.26791269e-01,2.11463168e-01],[3.97954509e-02,-5.79996645e-01,2.24745959e-01],[-1.20153576e-01,5.75366199e-01,1.93827674e-01]],[[-8.66981864e-01,2.84845918e-01,8.87006894e-02],[6.29381716e-01,-2.63265222e-01,-4.07063812e-01],[5.86426914e-01,3.04380983e-01,-2.25833938e-01]],[[-2.97845483e-01,-4.13520604e-01,1.67659789e-01],[4.91415411e-01,-8.24021757e-01,-5.44732153e-01],[-6.46015644e-01,-4.86376733e-01,1.14922695e-01]],[[-3.80104005e-01,-7.47050285e-01,-6.39769807e-02],[-7.16607869e-01,-7.43923843e-01,-2.39516482e-01],[-5.70686638e-01,-2.50042886e-01,-3.85651201e-01]],[[5.52435040e-01,-2.44337507e-03,5.76374590e-01],[-2.13374838e-01,3.82570885e-02,-6.51765168e-02],[5.11661433e-02,3.93424911e-04,-2.47838181e-02]],[[3.48887801e-01,-5.19105256e-01,3.56042653e-01],[-4.83286262e-01,-5.50458908e-01,8.90327431e-03],[-2.82276273e-01,8.16372395e-01,9.24343824e-01]],[[-1.81104243e-01,5.02501845e-01,-4.56415832e-01],[-8.98362175e-02,7.41905510e-01,2.72065043e-01],[4.27907892e-02,-4.98936296e-01,-7.93502808e-01]],[[-5.89911520e-01,-5.94739377e-01,2.27707803e-01],[-2.81752259e-01,-4.19405043e-01,-5.03220618e-01],[8.47043455e-01,2.65229315e-01,-1.25855148e-01]],[[-1.95022091e-01,3.20910811e-01,5.37144721e-01],[6.45365119e-01,2.87435293e-01,-3.80361587e-01],[-7.52943873e-01,-3.88955951e-01,5.57308719e-02]],[[1.01615250e+00,-3.85847837e-01,-6.48844894e-03],[5.63801229e-01,7.73340106e-01,2.88392276e-01],[7.77576506e-01,-7.54710555e-01,2.48234555e-01]],[[-7.15225279e-01,-6.11370564e-01,-3.34454924e-01],[3.39717627e-01,-1.24276668e-01,7.20808730e-02],[-1.23888582e-01,-7.22067058e-01,-1.09194887e+00]],[[7.56596029e-01,5.54913163e-01,5.29029548e-01],[-5.57548165e-01,-6.03524223e-02,1.08226568e-01],[-5.30808747e-01,2.81387568e-01,2.22064313e-02]],[[4.20010760e-02,6.09888732e-01,-3.38086396e-01],[7.75143385e-01,3.78370643e-01,-5.71192801e-01],[-3.28516901e-01,-5.77621281e-01,6.64694011e-01]],[[-2.95403264e-02,3.01580191e-01,1.36978641e-01],[5.02059519e-01,-7.78310180e-01,2.79611796e-01],[-2.68144682e-02,8.63468181e-03,-2.79801309e-01]],[[5.20423234e-01,-1.17157251e-01,7.31314626e-03],[8.94239068e-01,-2.18337625e-01,1.21930882e-01],[-2.55818907e-02,-4.28173654e-02,1.09628248e+00]],[[7.70504653e-01,-1.11090757e-01,-2.06362292e-01],[2.56535821e-02,7.27544576e-02,6.53423905e-01],[-3.35271448e-01,2.90300827e-02,-6.88702583e-01]],[[-1.20729601e+00,7.80230343e-01,3.54841024e-01],[-3.65712315e-01,2.81399608e-01,4.92598176e-01],[4.00195420e-01,-3.11684042e-01,-6.33156359e-01]],[[-2.06458822e-01,-3.57166529e-01,4.63722557e-01],[-3.00372183e-01,5.11590123e-01,-3.92178893e-01],[1.04799859e-01,2.93244291e-02,6.15584441e-02]],[[-3.26433480e-01,2.28601664e-01,1.61443323e-01],[-4.28886227e-02,-6.60517454e-01,-2.95926899e-01],[5.76206684e-01,-7.52571285e-01,1.54133308e+00]]],[[[5.58198512e-01,4.15831059e-02,-8.62576216e-02],[4.08207655e-01,2.95585901e-01,-2.33046040e-01],[2.55512685e-01,-2.24301443e-01,5.82436845e-02]],[[-4.58078265e-01,1.68998493e-03,-3.00071210e-01],[1.98647738e-01,-1.47909313e-01,-6.45246625e-01],[-1.34374633e-01,-1.24386393e-01,-6.16856992e-01]],[[-2.95344770e-01,5.85485756e-01,4.83709669e+00],[-3.49488914e-01,-4.88546997e-01,-4.22947593e-02],[-4.26375121e-01,-6.49093866e-01,5.00223875e-01]],[[2.53084630e-01,-1.32875308e-01,1.86798926e-02],[4.00733110e-03,-1.01744163e+00,3.57420519e-02],[-4.75218266e-01,3.93755227e-01,7.51222819e-02]],[[-6.78540230e-01,2.04884291e-01,-5.74296772e-01],[8.45851842e-03,2.28949681e-01,-4.42290574e-01],[-5.01438439e-01,-1.53473288e-01,-3.82080793e-01]],[[2.14741975e-01,2.85686076e-01,-3.46374214e-01],[-2.22455502e-01,-2.51148478e-03,-5.49137592e-01],[2.93728232e-01,7.55308509e-01,9.41280007e-01]],[[-1.50307968e-01,4.89622727e-03,3.37033242e-01],[-9.88316536e-03,4.41482395e-01,7.64550567e-02],[-3.64454061e-01,-3.86132412e-02,-4.23657754e-03]],[[5.34279644e-01,-3.16354841e-01,7.15728462e-01],[-6.32132173e-01,-3.25290442e-01,7.33699262e-01],[-3.21401358e-01,-2.45156780e-01,9.72457081e-02]],[[-3.68042439e-01,-5.66836372e-02,3.60809594e-01],[-9.23161089e-01,3.19734335e-01,-3.85940552e-01],[-9.64144617e-02,4.73367602e-01,-2.91769747e-02]],[[3.15954775e-01,3.31909746e-01,3.62863302e-01],[9.65091884e-02,6.07288897e-01,2.45601103e-01],[2.31129304e-01,3.55753154e-01,-3.39038670e-01]],[[2.34386161e-01,9.64269275e-05,-7.77874172e-01],[2.86496967e-01,1.70540214e-01,-1.12100601e-01],[-4.13566917e-01,2.89965510e-01,-9.98471975e-02]],[[-7.41348490e-02,-3.59636664e-01,1.57744324e+00],[1.19748242e-01,-4.33194488e-01,2.93597821e-02],[-2.08288729e-01,-1.62422493e-01,-6.13640994e-02]],[[-2.37797573e-01,-3.49715918e-01,-7.12227166e-01],[-6.38407707e-01,-4.74376321e-01,-1.97879538e-01],[2.19812497e-01,-6.05394661e-01,3.40760797e-01]],[[5.28928459e-01,-7.08514303e-02,-2.22428858e-01],[-4.14119624e-02,-3.80115092e-01,-3.25951248e-01],[2.26495981e-01,-7.20336959e-02,-2.07403004e-01]],[[-1.77294850e-01,-5.48661411e-01,3.60076465e-02],[-1.09508574e-01,2.53107041e-01,1.85195506e-01],[1.82604149e-01,3.30878980e-02,-3.94088835e-01]],[[3.66427898e-01,-3.64212483e-01,-1.22500765e+00],[-9.97411758e-02,-2.39861786e-01,-4.00983602e-01],[-4.11685824e-01,-5.30745327e-01,-3.33895534e-01]],[[6.49078965e-01,-3.28859419e-01,-2.57548392e-02],[3.78481776e-01,-5.57059765e-01,1.67526096e-01],[2.81488508e-01,2.35285133e-01,2.57776260e-01]],[[-3.62581670e-01,-3.21759224e-01,-5.99180877e-01],[3.26907098e-01,-5.75560749e-01,-3.31500441e-01],[-3.70773315e-01,-4.10802782e-01,-1.36310115e-01]],[[4.60625082e-01,-2.12089539e-01,-4.00434174e-02],[2.48405822e-02,7.16975182e-02,2.94577390e-01],[6.44279897e-01,-7.81280994e-01,4.50376600e-01]],[[-1.14649028e-01,2.26412609e-01,-5.35337806e-01],[-1.32833524e-02,-2.50383586e-01,5.63290238e-01],[-1.36034638e-01,-4.39967155e-01,-3.55637997e-01]],[[-1.12515353e-02,1.13231838e+00,1.11992812e+00],[-8.55836049e-02,-6.09033763e-01,9.24391389e-01],[4.13337797e-01,2.25609243e-01,-1.31073847e-01]],[[6.62342533e-02,-2.07300540e-02,-5.03329396e-01],[1.72438383e-01,1.47655774e-02,4.13920693e-02],[-1.12305611e-01,-3.97638083e-02,3.98020409e-02]],[[-9.76874456e-02,2.20174730e-01,1.33018827e+00],[-1.16766924e-02,1.84768438e-01,1.82021007e-01],[1.24435060e-01,-4.37408872e-02,-3.08315992e-01]],[[-3.25652987e-01,1.34629488e-01,-1.36108077e+00],[2.47315183e-01,-1.41473506e-02,1.35718197e-01],[-4.07564074e-01,-7.36337453e-02,-3.36963743e-01]],[[1.23922534e-01,-3.93673964e-02,-6.13154948e-01],[-5.24902284e-01,-1.81531951e-01,-5.21582127e-01],[-2.90609092e-01,-4.75816466e-02,3.70189369e-01]],[[1.36589140e-01,4.22455668e-02,3.28810692e-01],[-3.20144296e-01,4.30745870e-01,2.46268418e-02],[5.63717782e-01,5.33713102e-01,-9.59947333e-02]],[[-3.47239971e-01,2.27917895e-01,3.48829813e-02],[-6.92594826e-01,-3.54589164e-01,9.18762162e-02],[5.89761436e-01,5.67514062e-01,6.22136772e-01]],[[2.44594336e-01,-2.76505888e-01,-4.93974984e-01],[-1.04570486e-01,-5.27654827e-01,5.79097390e-01],[4.41273034e-01,-3.31270516e-01,-1.20288777e+00]],[[-1.99022785e-01,-2.54066467e-01,-1.28468597e+00],[8.68870169e-02,-1.70274884e-01,5.15458621e-02],[7.12977126e-02,-3.36480439e-01,6.60039932e-02]],[[-4.87780496e-02,-8.20314884e-01,1.28017318e+00],[-1.70945764e-01,1.92888409e-01,9.89385784e-01],[1.45433098e-01,3.52046698e-01,-1.63995326e-01]],[[-2.97966540e-01,7.89542854e-01,3.98681462e-02],[1.35224938e-01,-2.64778644e-01,-1.22089982e-01],[8.53988458e-04,-4.90067393e-01,-3.07793140e-01]],[[-2.63507038e-01,-1.79849893e-01,6.18536413e-01],[2.37863466e-01,-3.07127148e-01,-3.63805205e-01],[5.94465077e-01,3.15292925e-01,-8.35792959e-01]],[[-4.25581664e-01,3.76039505e-01,5.64285994e-01],[1.16748191e-01,2.54936755e-01,-2.32687369e-01],[2.93181509e-01,-5.24275839e-01,-2.38388851e-01]],[[-8.41881260e-02,-5.32948859e-02,5.56961238e-01],[5.38714826e-02,-6.84635192e-02,1.03080615e-01],[6.95507079e-02,3.67926240e-01,4.63961102e-02]],[[-2.53951520e-01,-1.48724616e-01,5.64726353e-01],[6.52402034e-03,-4.35398906e-01,-5.50155938e-01],[4.58398849e-01,4.07268494e-01,-1.71318531e-01]],[[1.58100218e-01,1.38383567e-01,-4.02033180e-02],[4.60557528e-02,3.99326742e-01,5.18720187e-02],[3.15738432e-02,-1.76703915e-01,2.75006384e-01]],[[-3.68099749e-01,1.90047935e-01,-2.71265864e-01],[2.93634802e-01,-4.11870539e-01,7.78195411e-02],[4.32595134e-01,-2.24165618e-01,-4.85226274e-01]],[[-6.05379008e-02,-1.83140218e-01,3.77463669e-01],[-5.18975556e-01,-1.70122176e-01,5.15968561e-01],[-2.16172531e-01,1.14309885e-01,6.62800372e-02]],[[-3.98128890e-02,-2.36090243e-01,-1.17306352e-01],[1.16954163e-01,-2.61180431e-01,1.51394859e-01],[-9.12621796e-01,-2.74611592e-01,6.35065287e-02]],[[1.54741749e-01,4.72539008e-01,3.59428674e-01],[-3.79519731e-01,-1.71234071e-01,-2.40624681e-01],[-2.80433539e-02,1.62746787e-01,-3.03827763e-01]]],[[[-0.02350359,-0.6577497,0.08137623],[-0.03893149,-0.18032023,0.14936309],[-0.04659261,0.05958358,-0.8080377]],[[0.10073372,-0.45176566,0.6262579],[0.4605713,0.23666438,-0.2384671],[-0.25279906,0.21741833,-0.02637887]],[[0.01556065,0.19624262,-0.14905629],[0.26468313,0.29561633,-0.2732006],[0.4093079,0.45587483,-0.42781347]],[[0.01885904,-0.47854427,-0.29611975],[0.01294439,-0.6421113,0.1361002],[0.04993783,-0.66101503,0.5514735]],[[-0.02616043,0.29604757,-0.9688942],[0.44539905,0.22241215,-0.4182019],[-0.5987102,-0.21849123,-0.77953833]],[[-0.41252857,0.38970983,-0.04611749],[0.02339431,-0.16775735,0.5012718],[-0.01453527,-0.18359359,-0.3439653]],[[0.24259089,-0.73976964,0.6704892],[0.68321204,0.0763701,0.01634044],[0.4002955,-0.62691677,0.44186646]],[[0.35171902,-0.54435855,0.16583613],[0.17548992,-0.522949,0.54560006],[-0.66253334,0.64398617,-0.3129975]],[[-0.27827924,0.10574228,-0.21969402],[-0.7487516,0.07741221,-0.35059762],[-0.2307161,-0.26929775,-0.10157943]],[[-0.42224187,0.2691781,0.3642497],[0.26801434,-0.66498494,0.614731],[-0.38950238,0.10645034,0.37928054]],[[0.35448623,-0.36461243,-0.12504326],[0.19353834,-0.7609886,-0.64418846],[0.03661357,0.4972935,0.4948758]],[[-0.59679556,0.15515955,-0.6404276],[0.6881351,0.55637276,-0.06709523],[0.38169035,-0.2872494,1.092356]],[[-0.38257268,-0.1156841,0.44498247],[-0.8731024,-0.9320639,0.06819861],[0.18323952,0.66229314,0.02564927]],[[0.28155315,-0.48305956,0.14342092],[0.39506853,-0.13611664,0.18312341],[0.02776967,-0.9458821,-0.46515238]],[[0.29256263,0.2253275,-0.66241723],[-0.20072015,-0.7744277,0.39958143],[0.7457159,-0.2877281,-0.8363043]],[[0.32979557,0.36919612,-0.2341152],[0.0434917,0.8472808,-0.5156502],[-0.16667174,-0.06514549,-0.10359906]],[[-0.81814104,0.17919146,0.25435084],[0.10458744,-0.8456102,-0.33405897],[-0.14566739,-0.3159267,-0.21878025]],[[-0.10185586,0.26853058,-0.84225774],[0.11155152,0.32329175,0.04212463],[-0.45276278,0.45596743,0.16787654]],[[0.07098854,0.24561457,0.19079056],[-1.1378278,-0.12658288,0.14310676],[-1.1278044,-0.4089532,0.41127905]],[[-0.30364504,-0.27620265,-0.2565346],[0.8681788,0.13685305,-0.16397229],[0.04621058,-0.43324274,0.27321994]],[[0.1074819,-0.24439827,-0.21114595],[-0.12243435,0.42092755,0.23664379],[0.1675282,0.26356554,0.07739408]],[[-0.00679328,0.24564189,0.63851285],[-0.09067564,-0.20334215,-0.64450866],[0.04294217,0.2805396,0.36044228]],[[0.1714582,-0.04922586,0.33040345],[-0.22324476,0.12578522,0.6981734],[-0.49802,-0.15301213,-0.03781837]],[[0.81948215,-0.23706076,-0.27262458],[0.8423656,-0.25318837,-0.8438334],[0.6680789,0.07578848,-0.38591996]],[[-0.04835593,0.13358319,0.00776623],[0.3585291,-0.7789418,-0.5101633],[-0.21626382,0.21459587,0.6717596]],[[0.4286393,0.27109578,0.22513515],[0.13320377,0.15173995,-1.13406],[-0.2529216,-0.79625726,-1.0408193]],[[0.5734788,-0.4044353,0.20774695],[0.11548349,-0.83655304,-0.62688184],[-0.61892945,-0.06883278,-0.40437]],[[-0.12335857,0.13976187,0.3487988],[-0.53698194,-0.25231057,0.27858606],[-0.26749906,0.71159244,-0.75653106]],[[-0.18606831,-0.5263625,-0.10913367],[0.2676844,0.04639279,0.575817],[-0.5008654,0.546228,-0.7441237]],[[-0.18724073,-0.42845124,-0.34755263],[-0.12267952,0.18900363,0.15380017],[-0.09490527,-0.14550827,-0.25348166]],[[-0.08862464,-0.21749946,-0.24131276],[-0.10491864,-1.1598068,-0.7092886],[0.62179947,-0.50910276,0.04923973]],[[0.40825617,-0.09983843,0.11423822],[0.53115106,-0.10455585,-0.26115555],[-0.5363513,0.0506204,-0.22121857]],[[-0.315252,-0.05736561,-0.4183613],[-0.32177573,0.1102193,-0.08406886],[0.04876792,-1.3603917,-0.54630655]],[[0.545738,-0.4069268,-0.61518514],[0.5685802,-0.46464118,-0.47760302],[0.07278294,-0.14653732,-0.15887775]],[[0.6074903,-0.19930597,-0.90884465],[0.91695553,-0.24223867,-0.32021588],[-0.11404842,0.11715236,-1.1501985]],[[-0.1961881,0.14938484,0.18062845],[-0.80861074,0.18433104,-0.12121836],[0.02960304,-0.75079095,0.38248584]],[[-0.27787974,-0.48991585,-0.00440976],[-0.00308193,-0.10026179,-0.32290646],[-0.29135552,-0.00309652,0.6728938]],[[0.42915663,0.1987422,0.852816],[0.26777026,-0.45109722,-0.03115939],[-1.0430535,0.27784166,-0.02712435]],[[1.0867573,-0.20100163,0.3863798],[-0.12959425,-0.03294575,-0.7306131],[0.41325378,-0.28994554,-0.508435]],[[-0.91332746,0.87590057,0.42742363],[-0.5997641,-0.21382418,-0.32783532],[0.72323334,-0.04224008,-0.14955859]]],[[[0.18052743,0.18335848,0.10199992],[0.07731507,-0.235812,-0.317721],[-0.4875399,-0.00228308,-0.43410435]],[[0.01984136,0.32662988,-0.34612045],[0.33407977,0.20807219,0.18186173],[0.36774284,-0.3466094,-0.1010593]],[[0.15384519,0.3724439,-1.3661758],[0.14690468,-0.03251265,-0.45708394],[0.04117247,-0.32788387,0.41903254]],[[-0.11278372,-0.5201647,0.4546692],[-0.47843167,-0.6709437,0.38465437],[-0.30771455,0.08446739,0.2827943]],[[0.1113946,0.15608568,-0.12150509],[-0.13945566,-0.36903498,-0.13964722],[-0.56379735,-0.05628916,-0.11293645]],[[0.7613957,0.27319634,0.74808574],[-0.25510758,-0.02479095,-0.6216898],[0.9650967,-0.20154445,0.44261128]],[[0.12888542,-0.07534837,0.3678804],[-0.02213695,-0.00649956,0.6551858],[-0.19272624,-0.1300279,0.01521504]],[[-0.01717015,0.52522343,-0.5899231],[-0.40007934,-0.323543,0.49341047],[-0.19506612,-0.81900334,0.20714715]],[[0.7317123,0.21920995,-0.43695486],[0.95551497,0.12397886,-0.13038827],[-1.3006183,-0.21443734,0.25883004]],[[0.46404907,-0.58353937,-0.068409],[-0.28829384,-0.8747559,0.3015356],[-0.03709101,-0.53019786,0.76722336]],[[0.09442932,0.00885202,-0.336292],[-0.19711466,-0.28721038,0.86097443],[-0.18107712,0.47680596,-1.0777119]],[[0.72250575,0.34285954,-0.68772626],[-0.58409745,0.4619027,-0.23518182],[-0.40245202,-0.33098137,-0.21622142]],[[-0.13752705,0.33338836,0.2826266],[0.6724207,-0.32087055,-0.15294024],[0.4078181,-0.61127377,-0.09517045]],[[0.3024561,-0.18064237,0.28930667],[-0.31584466,0.32468373,-0.01220391],[0.8279087,-0.12076905,0.4995411]],[[0.31188583,-0.35353303,-0.726128],[0.08784132,-0.34604686,-0.7346232],[0.8292342,-0.5450269,0.01684255]],[[-0.03095785,-0.63699234,0.1036028],[0.6069122,0.05140357,-0.22675116],[-0.16533943,-1.168723,-0.05325681]],[[-0.0867776,-0.43202743,-0.85082984],[-0.19044034,-0.9136908,-0.5107076],[-1.1878655,-0.22968233,-0.21052766]],[[0.3078327,-0.21035986,0.01102252],[0.0152601,0.87661356,-0.38195956],[0.26289746,-0.34148243,0.4324356]],[[-0.2861634,0.08119632,-0.31546676],[-0.41263586,0.6252133,-0.9252107],[-0.15089127,-0.8796057,0.22586577]],[[-0.38323092,-0.5025203,0.5458851],[-0.14327398,-0.19267191,0.576678],[-0.41952226,-0.5590319,0.3205663]],[[-0.36011615,-0.1535205,-0.48784083],[-0.20591271,-0.56764776,0.6683374],[0.7328208,0.12140378,-0.13574047]],[[-0.13483815,0.11456173,-0.22714235],[0.6913209,0.5733372,0.31350374],[0.6807959,-0.15588275,-0.10737032]],[[-0.10446484,-0.6431078,0.33367813],[0.2135647,-0.26909167,-0.07866936],[0.51287144,-0.04238867,0.43191683]],[[0.24324277,-0.36528337,0.9320981],[-0.47307938,-1.160409,-0.4174756],[-0.39572567,-0.35515025,-0.18674108]],[[0.15959458,-0.7095884,0.17745976],[-0.19316228,0.8952117,-0.2431185],[-0.06325718,-0.09338254,-0.32258925]],[[-0.21751009,0.7613957,0.12332272],[-0.375622,-0.45667845,0.7243916],[-0.5428486,0.21197204,0.35500267]],[[0.46617082,0.02280167,0.83886826],[0.31824568,-0.40888754,0.26959562],[-0.08208407,-0.6795373,-0.14145401]],[[0.24118665,-0.14377299,0.32676655],[0.40448266,0.5760529,0.43966132],[-0.6932721,0.42958927,0.62880147]],[[0.14262336,0.54991865,0.6984123],[0.68304193,0.5787939,-0.14558098],[-0.20556934,-0.35054132,0.27665645]],[[0.47043023,0.3405894,0.26469916],[-0.26783288,-0.58908296,-0.9902015],[0.41812408,0.45140094,-0.37953454]],[[-0.30795652,0.32176965,-0.12466461],[-0.31989515,0.38660297,0.14310563],[0.1694789,0.16148926,-0.04463268]],[[-0.34772658,0.06900559,-0.20116511],[0.17709431,0.57432467,0.42282376],[0.01254321,-0.3838695,1.06707]],[[0.2197704,-0.38609695,1.0686356],[-0.4797899,0.97962385,0.16065563],[-0.08589625,-0.09157987,-0.29145163]],[[-0.627458,0.5652771,0.64198405],[0.00272652,0.4916837,0.77485573],[-0.47147286,0.30023447,0.07908917]],[[0.6307544,-0.00938321,0.4057774],[-0.08419707,0.19349353,0.07468943],[0.88707674,-0.69355303,-0.45556596]],[[0.6844823,-0.7331171,0.02222206],[-0.00730784,-0.15777525,0.01629108],[-0.00268846,-0.7344837,-0.4018899]],[[-0.19955005,-0.19814523,-0.37835264],[-0.6159972,0.60363424,-0.06827377],[0.13997501,-0.18610519,-0.3098544]],[[-0.12072951,0.02948356,-0.2247834],[-0.60605794,0.71517277,-0.53463537],[-0.7977644,-0.15356046,0.35328043]],[[0.62471545,0.24361598,0.9759442],[-0.7528235,-0.00656749,0.3321731],[-0.20476823,-0.2565167,-0.39440763]],[[0.07322721,0.9154773,-0.7512045],[0.1626081,0.25522035,0.30364567],[-0.5725121,-0.43888158,0.06002499]]],[[[-3.50963145e-01,1.70309409e-01,5.28714597e-01],[-1.17459282e-01,5.20192087e-01,-6.30010739e-02],[-1.95207044e-01,-3.56651723e-01,-1.93515375e-01]],[[6.41616583e-01,-2.85370857e-01,-2.21896768e-01],[-1.56119838e-01,9.15947184e-02,-2.88958371e-01],[-3.99277866e-01,8.32627356e-01,-2.48163283e-01]],[[-1.11853719e-01,2.10900083e-01,-2.38015637e-01],[-2.03641807e-03,-4.27144021e-01,-8.08924958e-02],[-4.07191306e-01,5.41585758e-02,-1.30861491e-01]],[[-1.07560508e-01,-2.78621256e-01,1.64702684e-02],[-4.44679022e-01,1.48529157e-01,-1.82503507e-01],[3.38454068e-01,5.32076955e-01,-2.33641148e-01]],[[2.42832899e-01,2.15310737e-01,1.01361409e-01],[-3.98260474e-01,3.33938092e-01,-6.27801120e-01],[2.73727834e-01,3.99162740e-01,2.78697193e-01]],[[4.05381799e-01,5.41429110e-02,6.57321751e-01],[1.94126256e-02,4.31433022e-02,-9.40032423e-01],[6.22724175e-01,-4.38159108e-01,1.64591670e-01]],[[-2.11264208e-01,2.94920772e-01,-2.92055219e-01],[-3.08449656e-01,2.29709744e-01,5.75515032e-01],[-7.51753449e-01,8.00643042e-02,-6.57185586e-03]],[[2.10260287e-01,7.50299811e-01,4.08399612e-01],[-3.19482476e-01,1.27423465e-01,8.23924124e-01],[-1.06804597e+00,2.28634551e-01,8.12111259e-01]],[[4.72348779e-01,7.03569874e-02,-5.19738197e-01],[2.57043988e-01,-2.63635963e-01,4.83293861e-01],[-1.46159694e-01,8.36724788e-02,-9.12084162e-01]],[[4.69039589e-01,-2.60203391e-01,6.20729387e-01],[-6.43407643e-01,-5.56536727e-02,-7.94119418e-01],[-4.64610457e-02,-1.25199363e-01,-4.21517760e-01]],[[-2.09959224e-01,2.03457847e-01,-2.13247404e-01],[5.30092835e-01,-8.42191800e-02,-9.69598964e-02],[8.36460114e-01,-9.60977435e-01,8.98141563e-01]],[[1.20078647e+00,-3.12640756e-01,-1.76929399e-01],[7.15534449e-01,1.79402903e-01,1.90569729e-01],[-5.06980002e-01,-9.35204849e-02,-1.84361294e-01]],[[3.57758105e-01,1.95212945e-01,8.45392823e-01],[-2.75624972e-02,-1.16661876e-01,2.11427361e-01],[-3.19790721e-01,1.20209694e-01,1.57994747e-01]],[[3.52648824e-01,4.90174353e-01,9.03055891e-02],[-2.48512208e-01,2.86666453e-01,-1.92527026e-01],[4.03120369e-01,7.30352223e-01,2.44920090e-01]],[[5.07339716e-01,3.75308335e-01,-4.47693974e-01],[-1.59099042e-01,4.18399960e-01,1.17376399e+00],[5.37769377e-01,3.70907724e-01,-6.00033939e-01]],[[5.36790371e-01,-5.53933233e-02,7.03513861e-01],[1.92768678e-01,7.24569708e-02,-6.59801066e-02],[9.83034074e-02,1.86615035e-01,-5.83684504e-01]],[[1.21433213e-01,-6.32963300e-01,4.74867195e-01],[-1.17665243e+00,5.56508079e-02,1.39707580e-01],[-2.25031674e-01,4.63324845e-01,-2.40439679e-02]],[[6.32779419e-01,-3.85476083e-01,9.43674445e-02],[-2.89445370e-01,4.35346961e-01,4.69332993e-01],[3.74212772e-01,-1.74131140e-01,7.33480096e-01]],[[5.40925980e-01,1.54078662e-01,-9.02873993e-01],[4.24802452e-01,-3.58211552e-03,5.04249632e-01],[7.21144751e-02,-3.69361192e-01,-1.35631585e+00]],[[-1.30367845e-01,2.89148748e-01,-8.59078228e-01],[-2.15556666e-01,-2.27306321e-01,-3.06441188e-01],[-9.39003587e-01,1.53204322e-01,-7.70893276e-01]],[[4.30819541e-01,4.44896132e-01,6.24797404e-01],[6.18227422e-01,2.82422841e-01,1.12510726e-01],[5.14411665e-02,-1.48392752e-01,-3.60615700e-01]],[[-1.43718255e+00,-4.96145338e-01,2.56097555e-01],[-1.28467107e+00,3.93723190e-01,1.95811853e-01],[3.67251225e-03,-5.29197514e-01,-4.74664450e-01]],[[-2.36151904e-01,-1.93031818e-01,-1.24253500e+00],[-4.39700782e-02,5.79249084e-01,-3.45287770e-01],[-2.05981553e-01,-5.20314217e-01,-6.62829638e-01]],[[4.15857524e-01,3.48127574e-01,-4.36731815e-01],[4.45532560e-01,3.55661154e-01,-2.59369761e-01],[5.40779471e-01,6.81964904e-02,-3.44581515e-01]],[[-4.01273340e-01,3.75246078e-01,1.66589394e-02],[-8.20609152e-01,-2.69151956e-01,-5.02025485e-01],[-1.60964504e-01,1.52815074e-01,-4.41670239e-01]],[[-1.78750351e-01,6.31201342e-02,-8.49473551e-02],[6.16226792e-01,-4.40165162e-01,-2.28914753e-01],[3.76878053e-01,-5.73118508e-01,-6.61051393e-01]],[[-3.42479885e-01,-3.86472605e-03,-2.74308622e-01],[-4.80857104e-01,-7.22659454e-02,5.54177403e-01],[1.36145175e-01,4.49623942e-01,2.43160248e-01]],[[6.86471999e-01,4.47807077e-04,6.25911713e-01],[-4.38893855e-01,-8.69600996e-02,1.57131463e-01],[1.18450499e+00,-3.52001160e-01,2.87091643e-01]],[[1.14646360e-01,3.25477093e-01,9.34309721e-01],[-1.26156831e+00,-4.44842190e-01,3.10023904e-01],[-3.76525044e-01,3.18692356e-01,9.26852942e-01]],[[3.05475235e-01,-2.49722227e-01,-2.61268318e-01],[7.68346936e-02,3.52866054e-01,-2.29944676e-01],[-1.72052234e-01,6.03713155e-01,1.84320696e-02]],[[-4.38351631e-02,2.26756513e-01,5.59858441e-01],[4.40656245e-02,-1.39442980e-01,-7.01663643e-02],[-4.19774026e-01,7.58843482e-01,-3.15920234e-01]],[[1.89635996e-02,7.66142905e-02,-5.61915934e-01],[-4.59465027e-01,-1.51032001e-01,-6.79690838e-01],[-7.54792273e-01,-2.08800174e-02,4.06725377e-01]],[[9.28999856e-02,-2.27541000e-01,8.06713104e-02],[-4.42338943e-01,1.10746108e-01,-1.36504039e-01],[2.34040633e-01,4.25104201e-02,2.60160863e-01]],[[-1.09827906e-01,-2.60606676e-01,2.16101065e-01],[-7.20519960e-01,9.72326621e-02,3.30272377e-01],[-8.47755522e-02,6.77597761e-01,5.29636741e-01]],[[1.03483260e+00,-1.47562706e+00,-1.65861156e-02],[3.99849147e-01,1.10411957e-01,-5.40356338e-01],[-7.34361112e-02,-8.66341352e-01,3.81678760e-01]],[[-4.06539232e-01,-8.24771166e-01,-2.99123704e-01],[2.47760210e-02,1.58407614e-01,1.38121188e-01],[1.05523318e-01,-1.53174132e-01,6.89846635e-01]],[[-6.85059190e-01,9.96844694e-02,-5.11914909e-01],[-1.88659161e-01,1.16348898e+00,1.00750566e+00],[1.04571581e-01,1.55475676e-01,-3.90132278e-01]],[[1.38342589e-01,1.46769941e-01,1.84665725e-01],[4.87804025e-01,-5.28613031e-01,-5.73566139e-01],[9.25932899e-02,1.54756889e-01,-2.32672334e-01]],[[1.09846485e+00,-5.50817788e-01,-5.44661403e-01],[3.59603941e-01,-1.62662297e-01,-3.88727456e-01],[2.72027161e-02,9.71077606e-02,-9.29477155e-01]],[[2.84404933e-01,-3.22834581e-01,-3.82502198e-01],[1.78270966e-01,1.88657269e-01,-6.33619875e-02],[-1.89623743e-01,6.29335344e-01,2.65169144e-01]]],[[[-5.89941621e-01,-2.86587358e-01,1.81237847e-01],[5.30318916e-01,-3.47159337e-03,-6.91683292e-02],[-3.17361861e-01,-5.74751377e-01,6.20269358e-01]],[[1.14961602e-01,-1.59627739e-02,-2.52602220e-01],[-7.59878397e-01,-8.84940922e-02,-3.64588916e-01],[2.60451496e-01,3.11707854e-01,-1.64338017e+00]],[[-8.40266883e-01,-1.39717624e-01,-5.52118361e-01],[-4.18938994e-01,6.84664547e-02,-2.89497852e-01],[4.08888787e-01,8.42265561e-02,3.41933161e-01]],[[9.17638183e-01,-5.99185944e-01,-3.63278419e-01],[-3.94736260e-01,2.76903272e-01,3.04809242e-01],[-8.08603421e-04,8.24954391e-01,-2.28143007e-01]],[[-9.89115238e-02,1.37047589e-01,1.01725817e-01],[1.58576667e-01,-6.05609655e-01,1.07512437e-03],[5.60872927e-02,-1.25007823e-01,5.34425914e-01]],[[-4.45086896e-01,1.81622684e-01,-5.91235220e-01],[-1.26409888e-01,3.99358481e-01,-5.19472957e-01],[4.84645665e-01,-2.12570354e-01,-4.83312488e-01]],[[-2.42724836e-01,-6.86887324e-01,-2.82006443e-01],[5.69153368e-01,-1.66181386e-01,-1.64029479e-01],[-2.96662062e-01,1.46801651e-01,5.07671654e-01]],[[1.73404709e-01,-7.18852356e-02,-3.48926969e-02],[1.82153866e-01,-3.70904863e-01,-6.70433342e-01],[-1.55695714e-04,-1.90932274e-01,3.12367566e-02]],[[-3.57191861e-01,-4.00732309e-01,1.70730516e-01],[3.72707397e-01,3.35103601e-01,3.20186764e-01],[-1.84423998e-01,6.16347909e-01,-1.07841623e+00]],[[-5.46090044e-02,3.50897759e-01,-2.31592655e-01],[-4.35344547e-01,-1.76449165e-01,3.01974416e-01],[6.69425249e-01,-6.20012842e-02,-1.28060746e+00]],[[3.55583429e-01,-6.50784150e-02,-2.00816259e-01],[-8.83454550e-03,-1.06538944e-01,2.38314956e-01],[7.46390641e-01,-2.65477836e-01,-7.36669004e-01]],[[-6.47162974e-01,2.53466696e-01,-5.61580211e-02],[-2.46620417e-01,-3.04290280e-02,1.20756581e-01],[-7.16209590e-01,-4.31565255e-01,-5.71289659e-01]],[[4.95484918e-01,-1.05159414e+00,4.51715291e-03],[-1.29809037e-01,-6.40474737e-01,-1.04528332e+00],[-8.16725567e-02,3.84983510e-01,1.14726615e+00]],[[-2.76579350e-01,8.46707150e-02,4.49112207e-02],[6.50753617e-01,-7.84079079e-03,4.59580839e-01],[1.35626152e-01,-4.73842233e-01,-3.24417293e-01]],[[3.65610391e-01,2.90213346e-01,-3.70734297e-02],[1.57927424e-01,1.97252810e-01,-8.86943638e-01],[-8.30206037e-01,1.75006732e-01,-3.62741768e-01]],[[9.17859301e-02,2.43466482e-01,5.80161214e-01],[3.94282937e-01,-1.14286300e-02,4.25322145e-01],[-1.94609016e-01,8.24761391e-01,4.44948941e-01]],[[-4.32708800e-01,-4.66656208e-01,1.35167882e-01],[9.79214013e-01,-2.21582040e-01,7.00306654e-01],[3.94272238e-01,-2.46331617e-02,-6.08404636e-01]],[[-2.07425639e-01,-4.65784729e-01,-2.40610525e-01],[-8.60233486e-01,-9.39077139e-01,-1.70949280e-01],[-1.71251129e-02,-2.43437231e-01,-1.17568135e+00]],[[-1.14432601e-02,-3.01340669e-01,-1.21683168e+00],[6.12650394e-01,1.03069380e-01,4.68294442e-01],[-1.21937573e-01,-1.96225733e-01,9.28734690e-02]],[[-1.69297561e-01,2.57465184e-01,-1.08471699e-02],[-1.75559223e-01,-4.11429942e-01,5.66038489e-01],[1.88544050e-01,5.96397638e-01,3.38795744e-02]],[[-9.05933008e-02,-3.44631493e-01,-6.10781550e-01],[-3.68312448e-02,4.49649207e-02,4.82772775e-02],[1.52697399e-01,5.35624087e-01,1.71189547e-01]],[[9.01053131e-01,1.54117927e-01,5.07423460e-01],[3.71493399e-01,1.23822287e-01,-2.92319596e-01],[7.31458604e-01,2.34148085e-01,-8.99078324e-02]],[[-5.82399786e-01,3.64685118e-01,-6.46465659e-01],[1.01195365e-01,-2.87269824e-04,-2.74136029e-02],[3.38461787e-01,1.86948672e-01,7.97810331e-02]],[[-3.93959373e-01,-5.55272698e-02,-1.53064653e-01],[3.69236201e-01,6.42515182e-01,6.31329477e-01],[2.25921616e-01,1.36041790e-01,-5.13475180e-01]],[[7.99546361e-01,1.60544544e-01,-4.85433102e-01],[2.77333975e-01,-3.14776570e-01,4.33137238e-01],[1.50342673e-01,-3.33500534e-01,4.02485669e-01]],[[-4.65822071e-01,-1.85824096e-01,-3.84710312e-01],[5.58357909e-02,-1.22694814e+00,1.35477853e+00],[-6.15787327e-01,-4.09740657e-01,8.21177363e-01]],[[2.38401413e-01,-1.18972346e-01,-2.16626469e-02],[-2.97913492e-01,7.71488070e-01,4.53655839e-01],[-7.77803659e-01,3.01914126e-01,1.08233047e+00]],[[8.18586573e-02,2.21527323e-01,5.07194251e-02],[4.07725275e-01,-4.96821135e-01,1.07404304e+00],[-1.01710446e-01,-6.28130627e-04,-2.12555781e-01]],[[-1.41700730e-01,2.12983750e-02,-5.16168594e-01],[1.73486665e-01,-4.03221585e-02,-5.27612448e-01],[2.28648796e-03,-1.86441541e-01,-3.10923696e-01]],[[-3.33245724e-01,3.84115465e-02,-3.50963384e-01],[-2.22091958e-01,-4.79242280e-02,2.89200187e-01],[-2.37094343e-01,2.60539412e-01,-8.46089870e-02]],[[-4.79426354e-01,-3.66663218e-01,-6.21946931e-01],[3.24497879e-01,3.44754785e-01,-3.20526659e-01],[-1.07083023e-01,1.72611341e-01,3.36698055e-01]],[[-4.57225502e-01,-1.08893633e-01,-3.39614540e-01],[-1.69849083e-01,-8.63418281e-01,-7.78825656e-02],[-3.39507282e-01,3.31474572e-01,-7.70285726e-01]],[[6.57647014e-01,-8.10493976e-02,-5.00927329e-01],[1.38069913e-01,-3.72225285e-01,-4.64478940e-01],[1.06681131e-01,-8.15024853e-01,-1.75173546e-03]],[[9.51838046e-02,-3.09117913e-01,-6.24915719e-01],[-1.85786262e-01,-3.54133606e-01,1.36952937e-01],[-1.98629335e-01,-6.67258680e-01,5.87850772e-02]],[[-1.50419801e-01,6.18472360e-02,6.39572442e-01],[6.23439364e-02,-6.90621376e-01,7.07263827e-01],[7.16115355e-01,3.49269897e-01,3.10661435e-01]],[[-1.37822688e-01,4.81073797e-01,-8.69335383e-02],[-8.41805618e-03,-5.54150939e-01,2.03315526e-01],[2.27623567e-01,-1.17593265e+00,-1.31461307e-01]],[[1.93720371e-01,1.40001044e-01,3.19780052e-01],[5.71132414e-02,-1.27442911e-01,-3.39767784e-01],[2.13639319e-01,2.59497374e-01,-4.52607781e-01]],[[3.62709045e-01,-3.04987252e-01,-3.50081146e-01],[9.57026958e-01,-8.74134079e-02,7.13147461e-01],[6.07462302e-02,-2.19826147e-01,1.04475045e+00]],[[3.45759451e-01,1.03296220e-01,-1.96461111e-01],[2.76275128e-01,3.02010119e-01,-5.20892680e-01],[-1.78778589e-01,-7.33295858e-01,5.10380387e-01]],[[1.55890286e-01,-9.44014639e-02,-2.27129430e-01],[3.82158846e-01,-1.26054966e+00,-2.79496431e-01],[-2.30295911e-01,-5.18655889e-02,-1.17970717e+00]]],[[[0.7059943,-0.13207303,0.23204458],[0.68068856,-0.51457286,-0.06380514],[1.2595762,-0.15419887,-0.18836775]],[[-0.09909021,-0.08770867,-0.07792827],[-0.69492054,0.9166379,0.13527782],[0.4360685,0.4833366,0.3688624]],[[0.42253292,-0.2645944,-0.17664942],[0.32408014,0.24590982,-0.18415886],[0.03394847,-0.3872443,-0.53751737]],[[-0.21322496,0.5613333,-0.38110405],[-0.4186564,0.88742906,0.98931944],[-0.45437762,0.8090011,-0.37762934]],[[-0.16059856,-0.03587019,0.33002815],[0.28087497,0.42892942,-0.44174522],[-0.62066144,-0.07227706,1.0237473]],[[0.8534855,0.36447084,-0.38459232],[-0.10672574,-0.14124659,1.0610554],[-0.00291345,-0.6935656,-1.0233481]],[[-0.40048298,-0.24327508,0.32427776],[-0.48175305,-0.22424398,-0.20628044],[-0.12595302,-0.21359062,-0.7108548]],[[0.4744051,0.2242751,0.30988553],[-0.09625675,0.12756796,-1.1253382],[-0.23984349,0.65702075,0.04061041]],[[-0.13487414,0.65230304,-0.6095189],[0.04808506,-0.6460994,0.1630015],[0.18960276,0.17341726,0.25678587]],[[-0.7021607,0.38090003,-0.2880483],[0.22083732,0.42137477,-0.07409759],[-0.14961866,-0.13474548,-0.65098995]],[[-0.14446501,-0.190767,-0.12178732],[0.2979294,0.409376,-0.5207255],[-0.19190983,-0.04326666,-0.22052293]],[[0.08759995,-0.18694298,0.4033976],[0.27970284,0.9026915,0.01649861],[-0.32707015,0.64818585,0.20492004]],[[0.79891104,-0.3386607,0.28091106],[-0.29659882,0.5657134,-0.5509048],[0.02993807,-0.23064876,-0.05482795]],[[-0.6194885,-0.21276042,0.22083585],[-0.45572534,0.173223,0.30378202],[-0.04802951,1.157658,-0.18995015]],[[-0.24814585,0.7052787,0.06871147],[0.17331953,-0.08132809,0.05112265],[-0.74155325,-0.47728375,0.31066996]],[[-0.12757266,0.16378027,-0.6281864],[0.3756973,-1.3108621,0.42834595],[-0.32389992,-0.5245228,-0.11536603]],[[-0.41112003,0.26354715,0.14533861],[-0.7276896,0.7301276,-0.08502388],[-1.2003084,-1.0712785,0.43388352]],[[-0.04032445,-0.01769865,-0.2558285],[0.26583862,-0.19649422,0.67554533],[0.23536341,0.4637452,0.63326406]],[[0.05210495,0.1707024,0.18732004],[0.6234552,-0.32569337,0.46039942],[0.3677846,0.8828239,-0.26036438]],[[-0.16979097,-0.9136357,-0.41855448],[0.75131106,-0.09608214,-1.0145133],[-0.37863532,-0.8467939,-0.6271369]],[[0.1514364,-0.25123632,0.23183742],[-0.2504606,0.1800156,-0.11335504],[0.4137894,-0.16269378,-0.42722955]],[[-0.0522748,-0.65576947,-0.2012323],[-0.32368377,-0.6121415,-0.5312778],[-0.3025157,-0.3776888,0.10003942]],[[-0.18278404,-0.22775309,0.22759491],[-0.35400897,-0.3586395,-0.17943749],[0.18834546,0.57862896,0.48792887]],[[0.10449372,-0.7138857,-0.44022426],[-0.6197792,0.41759717,0.08378481],[-0.21107744,-0.36938256,-0.32490364]],[[0.1447205,0.31096703,-0.15908751],[-0.29666856,0.36019427,-0.36735186],[0.07709397,-0.45976844,-0.47607392]],[[-0.11843833,0.8752138,0.53502446],[0.10554136,-0.22112152,0.22655942],[-0.14936128,-0.98855305,-0.6723274]],[[0.55843973,-0.30564487,0.12030822],[-0.39703,-0.16983736,0.10300517],[0.29828438,-0.06732136,-0.89514965]],[[-0.30750096,0.31905413,0.05138326],[-0.19158286,-0.08647171,0.07062829],[0.9968701,-0.13723294,-0.69154924]],[[0.09265588,-0.57239217,0.23010223],[0.29970393,0.01923027,0.21999991],[-0.55452603,0.34542507,-0.5112348]],[[0.39557412,0.2761811,-0.6384427],[-0.08237237,-0.75915444,0.01499765],[0.00570168,0.76403046,0.07642104]],[[-0.6275929,-0.19440012,0.2870819],[-0.40066078,0.24551049,0.22170556],[-0.28909528,0.03346262,-0.16143042]],[[0.19413377,-0.16978155,-0.296784],[-0.19965585,-0.4115102,0.03036969],[-0.81116617,-0.3220561,-0.22249816]],[[-0.2920233,0.13476022,0.06862113],[0.28006095,0.29116246,0.06690097],[0.52754164,0.40961546,-1.0291563]],[[-0.7280013,0.0140622,0.6126747],[0.15172803,0.36716253,-0.45550716],[0.77290064,0.2280143,0.8656318]],[[-0.15071014,0.33366045,-0.38007453],[0.1178965,0.43285975,-0.38880605],[0.6152364,-0.13925016,0.05900199]],[[-0.02187995,-0.16808853,-0.13393721],[-0.11711732,-0.02476597,0.6251437],[-0.09429679,-0.46502325,-0.18032944]],[[0.97063804,0.21411443,-0.16554606],[0.48516002,0.46784222,-0.5397073],[-0.11905046,-0.0370025,-0.90516824]],[[-0.12678574,-0.02499984,-0.4700448],[0.52746016,0.17845252,-1.3290026],[-0.46864,-0.6420012,-0.26537406]],[[-0.03360332,-0.01747559,-0.24978285],[0.9599835,-0.65411836,-0.25636902],[0.04255711,-0.06613828,-0.3389889]],[[-0.4527977,0.643618,-0.10062763],[0.21603529,0.39795408,-0.37832755],[0.22573851,0.1959212,0.39650068]]],[[[4.55556214e-02,-3.32758754e-01,1.88275337e-01],[-1.96200415e-01,-3.28278840e-01,8.10906440e-02],[6.53039455e-01,-2.52034336e-01,-1.10113442e-01]],[[1.17885435e+00,7.21152306e-01,6.09791279e-01],[-3.05784374e-01,1.33298826e+00,1.22226492e-01],[-1.41558838e+00,6.29827455e-02,-1.09428239e+00]],[[-2.26329640e-01,-5.27742459e-03,-5.30485392e-01],[-1.52137861e-01,-1.96748286e-01,5.65134048e-01],[-3.72744262e-01,-3.78636941e-02,6.96540952e-01]],[[-4.36814785e-01,-1.20767629e+00,-9.49370623e-01],[-3.41941833e-01,2.76124984e-01,-5.34418344e-01],[5.52419007e-01,3.75896804e-02,2.82065541e-01]],[[-4.27732199e-01,-2.99569517e-02,8.61600190e-02],[-7.20482886e-01,2.29918584e-01,-3.12170178e-01],[2.15059102e-01,5.93295515e-01,-8.70421007e-02]],[[1.62749395e-01,-2.78132558e-01,9.45708528e-02],[5.84273577e-01,6.59280196e-02,3.42340827e-01],[-2.46902987e-01,-3.99485499e-01,-3.77124965e-01]],[[-8.41691375e-01,-8.64302665e-02,-4.54249457e-02],[-8.11286569e-01,-1.06897643e-02,-1.36592552e-01],[-4.76563513e-01,-2.22277969e-01,-3.73330116e-01]],[[-4.62296605e-01,7.20723152e-01,-8.68685663e-01],[2.56151110e-02,-5.51278070e-02,-3.26383024e-01],[1.11881030e+00,-4.36433077e-01,-2.10971564e-01]],[[1.89619195e-02,2.86894947e-01,-6.84751213e-01],[2.93578774e-01,4.35910851e-01,-3.25602382e-01],[-7.53730416e-01,-3.96484911e-01,-2.01030597e-02]],[[8.65100145e-01,4.69448179e-01,1.48416623e-01],[3.75820324e-02,3.53602976e-01,6.98852956e-01],[-5.03419816e-01,3.03973317e-01,-2.11092874e-01]],[[2.16104582e-01,-4.26462710e-01,4.66335297e-01],[-9.00481284e-01,2.76674539e-01,-2.86873370e-01],[-4.28107440e-01,3.51834059e-01,3.28574836e-01]],[[3.72334570e-02,-5.45305967e-01,-6.67693317e-01],[1.23036951e-01,-3.45093101e-01,-8.39918777e-02],[-4.90114316e-02,2.10149959e-01,-2.45417804e-01]],[[1.94475189e-01,4.30448860e-01,-2.85510659e-01],[1.27458966e+00,-7.95284271e-01,4.12532151e-01],[4.97508585e-01,1.76111341e-01,4.31159407e-01]],[[5.85869133e-01,3.44741940e-01,2.62539715e-01],[5.01222074e-01,4.36310917e-01,8.71856511e-01],[1.64544821e-01,-4.11073029e-01,7.27670044e-02]],[[-1.66676521e-01,-5.98444231e-02,-2.58020818e-01],[3.76324832e-01,-7.30197847e-01,1.55567229e-01],[-2.83887655e-01,2.71941513e-01,-6.16144538e-01]],[[-3.85237038e-01,-1.71992332e-01,1.63488463e-01],[3.95420760e-01,-2.24697199e-02,-2.13036254e-01],[2.53569465e-02,-1.70872584e-01,-4.21336234e-01]],[[4.24445480e-01,7.11801291e-01,3.54250312e-01],[3.45876127e-01,-9.81445983e-02,3.78301382e-01],[-1.63776308e-01,2.53388643e-01,-6.99285746e-01]],[[1.51850015e-01,-9.47186500e-02,-8.82103324e-01],[3.80025357e-02,-1.39568940e-01,-1.10211581e-01],[-8.73886347e-02,1.50039628e-01,4.10416871e-01]],[[4.43519533e-01,-1.37435660e-01,7.06417486e-02],[-7.52974331e-01,2.06642672e-01,-3.04642886e-01],[-9.62679505e-01,1.85074374e-01,-2.59764552e-01]],[[-5.54589406e-02,1.14647940e-01,-2.29016870e-01],[-5.83251655e-01,-1.20723672e-01,-2.27785826e-01],[2.97412246e-01,3.60039443e-01,-2.08404884e-01]],[[-1.07907757e-01,3.08443997e-02,2.38959134e-01],[-1.51346043e-01,1.73650041e-01,2.10548460e-01],[-2.80733202e-02,-4.77986842e-01,3.63941967e-01]],[[-6.27220273e-01,-9.58274007e-01,-3.39720577e-01],[9.44946557e-02,8.98377299e-02,2.36332282e-01],[-7.27449179e-01,5.63949168e-01,-3.79480779e-01]],[[-1.15686111e-01,-4.41501468e-01,1.71566188e-01],[-3.84591848e-01,1.27013892e-01,1.38892561e-01],[5.30386269e-01,-3.55268091e-01,8.18091273e-01]],[[8.16232026e-01,-3.55087012e-01,-5.24673727e-04],[7.04529822e-01,3.03965837e-01,-3.48771930e-01],[7.33608365e-01,1.49257537e-02,6.52174130e-02]],[[-7.19685853e-01,-3.14441204e-01,-5.00231624e-01],[-1.22179639e+00,1.66811675e-01,4.32392329e-01],[-3.23555857e-01,1.07743055e-01,-1.94520280e-01]],[[-4.25567180e-01,-2.26968318e-01,3.53911191e-01],[2.86564887e-01,-7.99927935e-02,-7.31262863e-01],[-5.97426713e-01,6.83551908e-01,-6.22131526e-01]],[[-1.21522106e-01,-5.99972121e-02,5.35027564e-01],[-3.31836373e-01,-1.91806972e-01,-1.89496204e-01],[-6.09516621e-01,-8.96703482e-01,-1.00397848e-01]],[[2.46800274e-01,1.55924812e-01,-2.19298467e-01],[6.05219722e-01,1.18239850e-01,3.25788289e-01],[-7.70572186e-01,3.70388359e-01,-7.31877506e-01]],[[-6.07875995e-02,-4.13379639e-01,3.07376925e-02],[-8.85823607e-01,-5.64428985e-01,1.68622255e-01],[-2.42955893e-01,-3.50435078e-01,3.66566986e-01]],[[4.91239756e-01,4.54868793e-01,4.51339483e-01],[6.77117527e-01,-6.18696630e-01,2.16389000e-01],[2.35223442e-01,-1.53635228e-02,-3.89957577e-01]],[[-1.80195551e-02,2.60739308e-02,-4.74081546e-01],[3.37707698e-02,9.54602838e-01,-2.67178446e-01],[-4.43613946e-01,5.45755289e-02,-1.26739472e-01]],[[-4.12612587e-01,-1.69534758e-01,1.68542296e-01],[-1.99482311e-02,-4.42107499e-01,7.20313311e-01],[-4.22295034e-01,-4.78727460e-01,-5.20963728e-01]],[[-1.22294627e-01,8.15074053e-03,1.00865233e+00],[-4.84977484e-01,7.13263303e-02,-9.01397094e-02],[-2.10813358e-01,4.57977861e-01,-1.47834584e-01]],[[-1.01559079e+00,4.77134138e-01,-2.32808106e-03],[-4.03768122e-01,-3.09031427e-01,-2.82769382e-01],[-1.51544988e-01,-5.05551755e-01,-1.26615927e-01]],[[1.06100082e+00,-3.61963287e-02,6.08840771e-02],[-2.71869749e-01,-1.89068973e-01,1.15452802e+00],[8.53986323e-01,9.89455521e-01,-1.68014750e-01]],[[-1.23349130e-01,-3.90780628e-01,-9.85958725e-02],[-5.55371106e-01,1.69010296e-01,-5.15209973e-01],[-4.67534661e-01,-8.85953128e-01,-1.07238568e-01]],[[-1.33173540e-01,-8.09515342e-02,1.99193031e-01],[-5.09579718e-01,1.06404670e-01,-3.06202978e-01],[-1.02348179e-01,-1.77163109e-01,1.56043554e-02]],[[1.42971473e-02,3.99073690e-01,1.51061058e-01],[-3.17127258e-02,2.61445194e-01,1.62740782e-01],[3.38373065e-01,3.94356847e-01,5.63713491e-01]],[[1.03619611e+00,5.19554138e-01,2.44716212e-01],[4.35576648e-01,1.28517315e-01,1.76729623e-03],[3.62823546e-01,3.27607036e-01,6.18646324e-01]],[[9.39907193e-01,6.19042218e-01,1.65237159e-01],[4.35327947e-01,4.60422575e-01,3.84592116e-01],[4.24051315e-01,9.60336685e-01,1.16238885e-01]]],[[[-7.61504173e-01,1.63601682e-01,-6.54634953e-01],[1.34530783e-01,-2.17865378e-01,2.36809403e-02],[-3.41483891e-01,3.78268808e-02,8.30512419e-02]],[[-2.72794932e-01,2.60616332e-01,5.58680296e-01],[1.43917367e-01,3.65544528e-01,4.97508198e-01],[-6.54141009e-01,5.09390235e-01,2.96075139e-02]],[[6.54700518e-01,1.53135523e-01,2.97883209e-02],[-2.29588225e-01,1.91880330e-01,1.37032270e-01],[-1.81030437e-01,-2.08695740e-01,7.13266313e-01]],[[5.90561144e-02,-1.98816255e-01,4.86423492e-01],[-1.23021409e-01,6.48494601e-01,-1.00774026e+00],[-2.72174567e-01,-1.33138178e-02,1.18812792e-01]],[[-3.63512874e-01,-7.14921132e-02,-5.31548679e-01],[1.84201419e-01,4.50585097e-01,-4.96952981e-01],[-2.02437043e-01,1.92654043e-01,-6.31831884e-01]],[[-1.19558528e-01,2.03252450e-01,3.56511436e-02],[-4.50255573e-01,-5.15644670e-01,5.55317044e-01],[7.99258769e-01,-3.63470763e-02,5.02381146e-01]],[[-4.54109490e-01,-3.26972663e-01,5.63410938e-01],[-4.27392542e-01,-5.86437106e-01,2.49427319e-01],[-4.17948812e-01,5.23597062e-01,1.16881043e-01]],[[7.80090988e-01,-2.65211929e-02,3.04224521e-01],[4.28983897e-01,5.40267348e-01,-2.60242432e-01],[1.13659604e-02,9.23207682e-03,-1.72518283e-01]],[[4.31627005e-01,-1.75226986e-01,5.37380934e-01],[3.86805236e-01,-2.45540097e-01,6.53261065e-01],[5.37474811e-01,2.55104184e-01,3.29408795e-01]],[[7.99355656e-02,3.12058985e-01,-1.47524416e-01],[5.39631486e-01,6.69915676e-01,4.97539863e-02],[4.95830864e-01,5.97343147e-01,4.31516171e-01]],[[-1.01382829e-01,1.26775011e-01,-5.28587475e-02],[-8.80791426e-01,2.35103190e-01,-6.58311367e-01],[-1.36236265e-01,3.81914020e-01,3.28050442e-02]],[[-1.27517683e-02,7.32558668e-02,-4.95334536e-01],[9.19341803e-01,2.33734429e-01,-1.10761218e-01],[3.14180076e-01,-1.48891419e-01,-2.47324422e-01]],[[-1.37578800e-01,4.61181432e-01,-5.69658056e-02],[-3.03099304e-01,8.37839246e-02,-4.77823347e-01],[-3.34784091e-02,5.31330228e-01,4.19367313e-01]],[[-4.09501225e-01,-5.14810048e-02,-1.97269730e-02],[-6.77229464e-01,2.47421294e-01,-5.37864685e-01],[-7.75743842e-01,2.81349748e-01,-2.04307050e-01]],[[-4.73567963e-01,1.01782143e+00,3.43323618e-01],[-5.98954976e-01,1.38040572e-01,-3.67446989e-02],[-3.52682084e-01,2.63331890e-01,-4.37795699e-01]],[[-1.84518889e-01,-1.04350261e-01,-3.47403198e-01],[-6.32697344e-01,3.16930085e-01,-7.50010669e-01],[-1.82428926e-01,-4.75400910e-02,-1.02623260e+00]],[[4.10315424e-01,-1.10496961e-01,-1.61034837e-01],[8.08854923e-02,6.62761807e-01,-5.25404751e-01],[8.61925125e-01,5.25536776e-01,-2.56507844e-01]],[[-7.32433319e-01,1.20917052e-01,6.69889003e-02],[-2.10220262e-01,4.17631954e-01,-7.33367130e-02],[-2.37601668e-01,-3.87558699e-01,-4.92964983e-01]],[[-2.78502703e-01,4.82603639e-01,1.36092246e-01],[4.75698262e-01,-6.28670216e-01,6.35266066e-01],[1.57308400e-01,-5.77810228e-01,8.78251731e-01]],[[3.44021142e-01,2.13282138e-01,2.31227338e-01],[4.29414868e-01,5.73871076e-01,3.41492236e-01],[3.79134923e-01,-2.89234489e-01,1.69967532e+00]],[[-3.59248340e-01,2.46740669e-01,-6.72906101e-01],[-4.79812205e-01,3.73869896e-01,-5.28605819e-01],[1.53531283e-01,4.86659631e-03,3.13321017e-02]],[[1.69774616e+00,4.06796604e-01,1.75708637e-03],[4.78561759e-01,3.73873740e-01,-2.11813790e-03],[4.18343008e-01,2.68882364e-01,1.98877696e-02]],[[2.34955862e-01,3.46834272e-01,2.17460513e+00],[-6.17969409e-03,2.84401566e-01,7.24839747e-01],[-1.22541040e-01,3.98664534e-01,3.54915977e-01]],[[-5.02590358e-01,2.65767545e-01,8.36163610e-02],[-4.24002916e-01,1.28784835e-01,-2.12743208e-01],[-5.22598088e-01,3.31713855e-01,3.92097741e-01]],[[7.63528585e-01,9.75046307e-02,7.02861130e-01],[6.41566873e-01,1.69676259e-01,6.81165040e-01],[1.21276915e+00,2.00409517e-01,9.23757911e-01]],[[2.06201106e-01,2.83348083e-01,2.91071571e-02],[-2.87518799e-01,-5.70906758e-01,7.44752511e-02],[-7.49445707e-02,-5.30834615e-01,5.03905654e-01]],[[9.58599150e-02,-3.36578578e-01,5.95913529e-01],[-3.61898750e-01,2.91671008e-02,6.28379166e-01],[3.55261594e-01,-3.54556710e-01,1.21789241e+00]],[[3.46187115e-01,4.84368891e-01,-2.53972993e-03],[5.48041642e-01,-1.77589685e-01,-3.20865065e-01],[-1.73405275e-01,2.10547745e-01,1.38101861e-01]],[[-5.85021257e-01,-9.86772627e-02,-2.14457965e+00],[-5.30131347e-02,4.11054164e-01,-5.72663009e-01],[-1.05230868e-01,3.65116782e-02,2.66016424e-02]],[[-4.34953779e-01,-6.62792623e-02,3.80577087e-01],[-3.59065950e-01,2.58011281e-01,-3.75607491e-01],[-9.93858516e-01,1.33653954e-02,-5.86647809e-01]],[[-9.54974711e-01,-8.21595022e-04,3.15036356e-01],[-3.63548428e-01,-8.85903910e-02,-7.89408311e-02],[-2.26147041e-01,1.84367880e-01,-4.51720059e-01]],[[3.53688635e-02,1.64274454e-01,-9.48575512e-02],[-2.22122848e-01,1.01515681e-01,9.50022995e-01],[1.58136770e-01,-6.90025151e-01,2.43140399e-01]],[[2.59408981e-01,2.30689958e-01,4.46291149e-01],[4.46775198e-01,-1.76253721e-01,7.15678871e-01],[2.96422392e-01,-1.65075198e-01,1.06131756e+00]],[[1.63867101e-01,-1.15162194e-01,1.54874623e-01],[1.37909845e-01,1.00257087e+00,-1.17241442e-01],[1.27620980e-01,-3.21629792e-01,-2.79874116e-01]],[[-1.60993159e-01,2.39166577e-04,-6.62679225e-02],[-6.40967712e-02,4.65230435e-01,-4.76367652e-01],[-5.87490618e-01,-1.32574439e-01,2.01910790e-02]],[[1.86811626e-01,-3.44898343e-01,3.09558474e-02],[6.98065534e-02,5.37176609e-01,1.64154574e-01],[-1.00101948e-01,5.13043284e-01,4.52100486e-01]],[[-4.73470777e-01,-4.18860018e-01,2.26660848e-01],[-1.00614107e+00,-9.26918238e-02,-9.29706916e-02],[-2.53856212e-01,3.60398963e-02,-7.94954076e-02]],[[3.33160490e-01,3.70409161e-01,-1.53487232e-02],[5.05868971e-01,3.84407938e-01,-1.82680503e-01],[9.86558080e-01,-2.44086727e-01,2.67884433e-01]],[[-5.17067730e-01,3.21038604e-01,-4.24235493e-01],[-1.28378212e-01,-7.08783984e-01,2.85761386e-01],[1.89150944e-01,-3.26996058e-01,7.74519295e-02]],[[3.47304642e-01,6.45112395e-01,2.29062870e-01],[1.01885654e-01,2.35353485e-01,2.64285445e-01],[3.36647362e-01,-2.74334103e-01,-6.49321616e-01]]],[[[9.80992198e-01,-1.86486930e-01,-1.00601219e-01],[-7.25695074e-01,2.25996412e-02,-3.40157628e-01],[-1.07052587e-01,5.24890125e-02,7.91500628e-01]],[[-3.96296024e-01,-6.87853545e-02,-2.48122662e-01],[-4.24972028e-01,3.34559232e-02,4.07773197e-01],[8.16214025e-01,-2.23324746e-01,4.80513066e-01]],[[7.59021401e-01,-4.11468983e-01,1.28395200e+00],[-2.20738322e-01,-1.50324434e-01,-4.36385162e-02],[-3.25793177e-02,-2.00292289e-01,-1.89815462e-02]],[[-5.94917357e-01,6.69704199e-01,-2.50204176e-01],[-5.28961897e-01,-2.46929109e-01,5.96545219e-01],[1.64509088e-01,1.98701367e-01,8.09083462e-01]],[[2.75121599e-01,4.47888583e-01,6.01808548e-01],[4.64567721e-01,1.36203477e-02,-2.54563987e-01],[-1.75792754e-01,3.96233261e-01,-8.00483897e-02]],[[1.84036851e-01,1.59659553e-02,1.08963668e-01],[3.09045404e-01,8.83640468e-01,3.60248923e-01],[-4.71798331e-01,-1.36204445e+00,-6.28200710e-01]],[[2.41322100e-01,3.56691927e-01,2.24052444e-01],[-4.44656193e-01,-6.50467336e-01,-1.22926414e-01],[3.48023415e-01,-3.18458021e-01,2.93631792e-01]],[[-6.07716918e-01,1.69566333e-01,-1.49717510e-01],[2.89165735e-01,5.23872793e-01,9.14786104e-03],[-4.28764708e-02,9.20620024e-01,2.08651647e-01]],[[3.68783146e-01,3.54740202e-01,-5.16099930e-01],[2.32441887e-01,4.67732668e-01,7.45243251e-01],[-4.36623506e-02,-5.94465137e-01,-6.49386823e-01]],[[-4.24091905e-01,-3.93645704e-01,4.46854085e-01],[-5.10144234e-01,-1.21597812e-01,3.10448557e-01],[-6.26428246e-01,-7.42606163e-01,-7.43107498e-02]],[[-3.28521788e-01,3.31844151e-01,1.55902728e-01],[-5.24840020e-02,3.33381027e-01,1.39598876e-01],[6.38574481e-01,8.75198007e-01,3.42481673e-01]],[[8.90411913e-01,-2.90533334e-01,-1.36076704e-01],[-1.44985160e-02,-3.14638168e-01,6.38060212e-01],[3.78144413e-01,2.69325346e-01,-8.15875411e-01]],[[-3.56570095e-01,9.48798656e-02,-6.66921139e-02],[-1.88081697e-01,4.87052679e-01,-8.62942100e-01],[-1.10832721e-01,-3.49010415e-02,-9.82927859e-01]],[[-5.49740314e-01,-2.30775461e-01,-2.85351649e-02],[5.55852890e-01,5.91575988e-02,8.09354484e-02],[2.30251625e-01,-4.84353304e-02,6.09761775e-01]],[[-3.83387148e-01,-4.11087364e-01,3.28495651e-01],[9.63960215e-02,3.52284491e-01,-3.88308346e-01],[5.02097011e-01,-8.30630839e-01,2.23316506e-01]],[[-3.73284340e-01,2.22771198e-01,-5.49479783e-01],[1.93792034e-03,-3.59006561e-02,3.80341262e-01],[-4.17790204e-01,-7.91722834e-02,-7.46426821e-01]],[[-5.78196406e-01,6.09577596e-01,6.44144773e-01],[6.39041603e-01,-5.06301641e-01,1.71328396e-01],[-6.03154242e-01,-8.61649960e-02,1.25545931e+00]],[[-6.69567764e-01,-6.05147362e-01,-3.75327468e-02],[-1.01146251e-02,1.91228732e-01,-3.79336834e-01],[-9.38673988e-02,2.28264287e-01,-1.80839822e-01]],[[-3.66699815e-01,-2.83516169e-01,-5.46342313e-01],[-2.81381875e-01,-4.29226786e-01,-1.82688504e-01],[-3.62105101e-01,3.25422019e-01,6.28290931e-03]],[[3.95085476e-02,-1.25446931e-01,-1.06659144e-01],[-3.10850710e-01,-6.83633089e-02,-5.59273899e-01],[-8.20667893e-02,2.33567655e-01,-3.72111529e-01]],[[-2.81881809e-01,-8.09742093e-01,-6.33824289e-01],[4.58174109e-01,-6.27578422e-02,2.97877222e-01],[-4.02057916e-01,1.02674139e+00,-3.27284157e-01]],[[7.89513767e-01,-7.17150122e-02,-1.65893465e-01],[1.00714028e-01,2.27439150e-01,-1.09674521e-01],[4.56812739e-01,6.56092986e-02,5.22115350e-01]],[[-3.02404255e-01,-2.68196195e-01,5.13263822e-01],[2.63840258e-01,-1.10657774e-01,-5.35970218e-02],[8.88198838e-02,1.30597681e-01,3.47355127e-01]],[[-6.95709050e-01,1.03787147e-02,-1.28228426e-01],[7.43233785e-02,3.34598750e-01,6.28946960e-01],[-9.84325334e-02,-4.27530855e-01,6.04446121e-02]],[[-8.70580256e-01,2.78007805e-01,-3.57534796e-01],[-3.22425544e-01,-1.19396098e-01,7.42762446e-01],[-1.20312743e-01,1.38900548e-01,-1.99178144e-01]],[[5.33344410e-02,-1.05142690e-01,-1.73304081e-01],[-2.63944298e-01,2.44262695e-01,3.35985929e-01],[1.49589336e+00,3.80914718e-01,4.80724305e-01]],[[7.69835472e-01,1.56121096e-02,3.52251112e-01],[-2.28626475e-01,3.41479361e-01,-1.85264096e-01],[2.62152791e-01,-8.60416472e-01,-9.13948536e-01]],[[2.28763446e-01,2.61579037e-01,2.84850001e-01],[3.13628018e-01,7.64130294e-01,2.81251743e-02],[5.55971920e-01,-1.19626284e+00,9.08997506e-02]],[[-3.45881909e-01,-2.87239611e-01,-3.31443608e-01],[1.15245618e-01,-1.44956762e-03,9.20684859e-02],[2.64614701e-01,-3.11500728e-01,3.76379639e-01]],[[1.27394259e-01,-2.99833924e-01,4.45819914e-01],[-4.68166173e-01,1.29271761e-01,1.38488436e+00],[8.02138820e-02,6.07991777e-02,-6.49061203e-01]],[[-4.07645464e-01,-9.75056067e-02,6.04595721e-01],[-9.95367467e-02,-1.10551894e-01,-9.28467198e-04],[-4.79106486e-01,-7.57843018e-01,6.29075468e-01]],[[7.19328344e-01,-4.15287793e-01,-4.64204187e-03],[2.20065922e-01,-1.14433694e+00,-3.75202835e-01],[1.09979868e+00,1.15901336e-01,1.03411937e+00]],[[-5.17789483e-01,2.20370501e-01,2.09661424e-01],[-5.38952291e-01,8.15865993e-01,9.60892677e-01],[1.63085923e-01,-2.24190012e-01,-3.43192279e-01]],[[9.14702788e-02,-2.64740944e-01,-1.18089221e-01],[7.29985237e-01,-4.56956506e-01,5.44557214e-01],[-6.32941961e-01,-6.49098575e-01,-5.39631963e-01]],[[3.54237676e-01,4.74045932e-01,-2.48044759e-01],[2.29118150e-02,-6.67137094e-03,6.32993817e-01],[4.26776886e-01,1.90438524e-01,-4.40428436e-01]],[[4.57947284e-01,3.45296502e-01,9.96698499e-01],[-1.35966927e-01,3.19222271e-01,1.31454736e-01],[-8.33135843e-01,-8.67291689e-01,-6.57531679e-01]],[[-4.87005651e-01,2.00011790e-01,7.37817705e-01],[-5.78214705e-01,2.08784267e-01,1.66837007e-01],[-7.04843178e-02,5.54576993e-01,8.14237654e-01]],[[5.00165164e-01,-2.01680496e-01,1.25429735e-01],[3.78147930e-01,-2.29713976e-01,5.54957211e-01],[-8.21015060e-01,2.51427889e-01,1.87496915e-01]],[[5.54404378e-01,-3.56898755e-01,-1.42546594e-01],[-2.83020169e-01,-2.26733983e-02,1.53027046e-02],[9.20926705e-02,4.06865090e-01,3.42407942e-01]],[[-4.99181867e-01,4.73897934e-01,4.72542077e-01],[1.31982625e-01,-7.99012780e-01,-4.95343953e-01],[-1.15799785e-01,-1.62288904e-01,3.35065901e-01]]],[[[2.83189774e-01,2.79850602e-01,2.92614043e-01],[6.30927905e-02,5.81293166e-01,-2.57020332e-02],[-5.25425792e-01,5.88162899e-01,1.62258774e-01]],[[6.94506168e-01,2.83762187e-01,2.03771397e-01],[-4.88624573e-01,3.55372667e-01,2.34559998e-01],[-1.17042887e+00,4.73075688e-01,-3.34457666e-01]],[[3.92407089e-01,-4.67209876e-01,-4.91231591e-01],[1.03762023e-01,4.43147719e-02,-2.21399114e-01],[-2.22117305e-01,2.03580961e-01,-2.98288524e-01]],[[5.81320643e-01,-4.74512637e-01,3.12134743e-01],[1.50370926e-01,-1.27911180e-01,7.15167299e-02],[-1.06769376e-01,7.63730705e-01,2.30249643e-01]],[[-2.17328891e-01,-4.97526191e-02,-2.69079179e-01],[6.55677736e-01,-5.04004024e-02,-2.09516525e-01],[4.39307630e-01,3.67210954e-02,-9.34160709e-01]],[[2.47518361e-01,-7.40468204e-01,-8.92857388e-02],[9.81466591e-01,-2.95228586e-02,1.60358697e-01],[2.80976027e-01,6.06993213e-02,-1.77345976e-01]],[[-5.58267593e-01,-5.53082645e-01,-8.29645574e-01],[-5.06868780e-01,2.17173263e-01,2.42559150e-01],[-8.01322103e-01,5.45694351e-01,6.71173275e-01]],[[2.24127233e-01,-6.51137888e-01,-6.65578127e-01],[2.69166291e-01,1.95526898e-01,4.52856123e-01],[1.63293570e-01,5.33032537e-01,-9.13431406e-01]],[[6.73957840e-02,-2.51943506e-02,-1.28989846e-01],[6.32779598e-01,1.11214972e+00,-2.60921180e-01],[4.57831055e-01,1.05173695e+00,2.50330213e-02]],[[-1.13758075e+00,1.16037548e-01,-2.88086355e-01],[4.54628207e-02,8.96483846e-03,-1.80703089e-01],[-7.01669514e-01,4.03515995e-01,3.55888575e-01]],[[-5.88994145e-01,5.01140654e-01,-7.55035460e-01],[-9.00729448e-02,3.14387769e-01,4.31949914e-01],[-3.80475551e-01,-4.17530775e-01,-2.86958754e-01]],[[2.45701984e-01,-2.85426199e-01,-2.19171301e-01],[-1.74704120e-01,-3.20439637e-02,3.53164464e-01],[-1.31655455e-01,-5.21605760e-02,1.76758841e-01]],[[2.26490334e-01,-7.75569916e-01,6.37967885e-01],[4.94697303e-01,-3.04405123e-01,-1.23896338e-01],[1.28305256e-01,1.09617278e-01,9.76750135e-01]],[[-8.49383891e-01,-2.27886587e-01,-2.91075289e-01],[5.34925520e-01,-3.46512049e-01,-1.11128855e+00],[-1.27476931e+00,1.78743571e-01,3.40778738e-01]],[[1.70218293e-02,-2.70730197e-01,-8.03991184e-02],[-1.34896159e-01,-4.48072478e-02,-1.31411791e+00],[5.59617281e-01,-6.64653838e-01,-1.84485227e-01]],[[1.27528682e-01,-8.06132406e-02,3.10271442e-01],[5.73875159e-02,6.36571586e-01,-9.64891493e-01],[9.23595428e-02,-2.95833886e-01,7.17106387e-02]],[[1.84650168e-01,-2.77372226e-02,-3.74948472e-01],[3.81844640e-01,1.01965312e-02,3.62216145e-01],[3.99041265e-01,1.19095564e+00,-5.93253933e-02]],[[2.64737636e-01,7.89846063e-01,3.60722512e-01],[1.08042300e+00,-2.10757349e-02,-3.63270789e-01],[1.27555296e-01,1.76580086e-01,4.31190193e-01]],[[-2.21433446e-01,-3.22421849e-01,2.40584373e-01],[-1.71500668e-01,-7.21714795e-01,-1.32689402e-01],[-3.88492823e-01,-7.71803856e-01,-2.65450895e-01]],[[-5.27929664e-01,4.69015688e-01,3.33606511e-01],[-7.52799988e-01,-1.11897523e-03,4.35364991e-01],[6.98307931e-01,-6.93536878e-01,5.77765942e-01]],[[-3.65720570e-01,-3.66171330e-01,-5.62788010e-01],[-1.50476843e-01,7.02701569e-01,-3.71271491e-01],[6.96095526e-01,5.23891225e-02,-1.17962666e-01]],[[1.68238711e+00,4.40381289e-01,-1.55474216e-01],[5.64396799e-01,3.99883151e-01,-5.71705818e-01],[-2.74457447e-02,-7.31194615e-01,-3.08007956e-01]],[[1.47920370e-01,3.62388551e-01,2.87067860e-01],[-2.22931821e-02,-2.75666505e-01,1.56042993e-01],[-2.35893667e-01,-4.37130541e-01,-1.49784759e-01]],[[-5.50334752e-01,3.02334487e-01,5.08393466e-01],[-9.03232917e-02,-9.20275748e-01,1.09251237e+00],[-4.75678980e-01,3.70270282e-01,2.80456185e-01]],[[-7.31361508e-02,3.53726596e-02,3.74145776e-01],[-3.62888068e-01,-1.75068438e-01,1.62490364e-02],[-2.05932096e-01,6.74940765e-01,4.11742032e-01]],[[-3.38065326e-01,-7.14319050e-01,6.87760860e-02],[6.07180223e-02,-3.40620309e-01,1.63841039e-01],[1.16874504e+00,-4.31827337e-01,3.85599196e-01]],[[-5.16119190e-02,5.92054307e-01,-1.52781054e-01],[3.92158329e-01,-1.82462439e-01,1.83856804e-02],[-4.14478362e-01,-8.74397531e-02,4.35618520e-01]],[[-3.26510012e-01,2.21314341e-01,-2.61256814e-01],[3.07797343e-01,-6.23998642e-02,3.01850468e-01],[1.03276062e+00,6.92937016e-01,4.26528871e-01]],[[5.65901995e-02,-5.47900319e-01,-3.39553952e-01],[-6.13917172e-01,4.09439445e-01,6.54584110e-01],[3.34441781e-01,-4.35424805e-01,-7.75707901e-01]],[[-9.82517526e-02,1.68680131e-01,-5.21089956e-02],[3.39365512e-01,-8.51098835e-01,-3.50048512e-01],[-1.96835279e-01,3.81875783e-01,-2.91422606e-01]],[[8.43016058e-02,1.51578024e-01,4.59854275e-01],[-5.36666453e-01,2.19842777e-01,2.33595595e-01],[1.94450676e-01,-4.97036040e-01,1.88760720e-02]],[[3.45845781e-02,3.87218386e-01,5.65895438e-01],[-8.12920108e-02,1.54100239e-01,1.55249506e-01],[5.01077294e-01,-1.52015701e-01,-9.80506659e-01]],[[-4.63636249e-01,1.52771071e-01,-1.62546396e-01],[6.04055882e-01,-1.26855537e-01,1.67745799e-01],[2.16500401e-01,7.85406411e-01,2.93903708e-01]],[[-4.53168191e-02,4.54424381e-01,6.96683675e-02],[-1.29470497e-01,2.64035195e-01,1.54210836e-01],[3.28256220e-01,-2.36775488e-01,-6.72348082e-01]],[[-1.29807860e-01,-2.89997399e-01,-1.45133600e-01],[-2.09089503e-01,5.18414140e-01,-8.27066600e-02],[-7.71501437e-02,3.73202920e-01,-9.43049788e-02]],[[1.65564150e-01,1.81457639e-01,-2.19309703e-01],[-4.67419535e-01,4.54318464e-01,-2.38835067e-01],[-5.31165421e-01,4.17970628e-01,2.00910479e-01]],[[5.29658258e-01,2.93202400e-01,-1.03343494e-01],[2.18074992e-01,-3.41204733e-01,1.94901854e-01],[2.27834135e-02,-7.48953894e-02,-2.93212056e-01]],[[-5.05747795e-01,-7.16563880e-01,-7.23325312e-01],[-2.02019885e-01,-2.61961091e-02,-3.30313593e-01],[6.35098100e-01,-4.44210857e-01,-3.32821786e-01]],[[-7.05431998e-01,1.70697913e-01,-5.57524323e-01],[-9.63296443e-02,5.49410462e-01,-2.57307202e-01],[3.04937456e-02,3.16241682e-01,1.15124404e-01]],[[6.43600464e-01,-5.69733322e-01,2.12075517e-01],[-9.39082131e-02,-2.88695514e-01,-5.69712102e-01],[-2.59448439e-01,2.28220791e-01,-3.23668569e-01]]],[[[0.37381458,0.7271486,0.35138398],[0.6844006,-0.7033985,-0.06380745],[-0.63417166,-0.01739228,-0.93164515]],[[-0.3135474,-0.32216406,0.40196297],[0.40842864,-0.33342594,-0.01257955],[-0.65291977,-0.19478162,-0.21905035]],[[0.23515597,0.0883445,-0.29618803],[0.18787515,0.01251325,-0.08161715],[-0.22857109,0.35766363,-0.04386597]],[[0.1626892,-0.6011148,0.52414715],[-0.25055772,0.47168368,0.4398013],[0.27552897,-0.39151308,0.327538]],[[-0.0737497,-0.57619524,-0.396725],[0.27296674,0.25054234,0.73307395],[-0.60887367,-0.47978672,-0.33573043]],[[0.5586681,0.00368837,0.14622569],[0.26941222,0.6121593,-0.6966101],[-0.22281678,0.33851317,0.27835315]],[[-0.14404002,-0.04547074,-0.05648379],[-0.36267835,-0.4409141,-0.27656135],[1.3842235,-0.29530454,-0.39207435]],[[-0.28733733,0.8492081,-0.31410798],[-1.0181894,0.3689961,-0.8307105],[-0.94629556,-0.12258141,-0.13146277]],[[-0.34291077,0.35846305,-0.3737822],[0.1405616,0.25241837,0.49746615],[0.35804293,0.82163346,0.26876193]],[[-0.11979455,0.5456067,0.14624229],[-0.5463809,0.40595633,-0.16426405],[0.05449858,0.41567287,-0.722899]],[[-0.00991126,-0.33723956,0.39796358],[0.17619509,-0.01939511,0.24250133],[-0.05440218,0.7071178,0.5114474]],[[-0.69311035,0.6470763,0.02352148],[0.43971702,-0.1738868,-0.72410417],[-0.5185115,0.10693964,0.6951877]],[[-0.80635023,0.92574143,0.54766595],[-0.29641226,0.06753594,0.04143615],[-0.55761397,0.8616684,0.55837697]],[[-0.29970878,-0.684515,-0.27462634],[-0.1656172,0.07629119,0.6183669],[0.17498474,0.5012739,0.01306347]],[[0.41809157,0.20237623,0.11511653],[0.23210528,-0.2826548,-0.4151069],[-0.27293038,-0.25639442,0.04595432]],[[-0.34141433,-0.22018732,-0.06627064],[-0.08450869,-0.5768562,0.24217878],[0.554431,0.48908988,-0.23039338]],[[-0.5122455,0.6201765,-0.32398948],[0.46287403,0.39085498,0.46406022],[-0.81895864,0.13554905,-0.8623683]],[[-0.2872848,0.37587565,0.05732828],[-0.27301845,-0.17021905,-0.09142585],[-0.20373665,-1.0847456,0.6388885]],[[0.3307799,-0.20878464,0.27498558],[0.3779764,0.39393288,-0.5688837],[1.1718528,-0.8082075,1.2499155]],[[-0.10561026,0.27546138,-0.19444926],[-0.21364094,0.22099431,0.37715995],[0.10975286,-1.0088758,0.3720893]],[[0.10526367,0.56040555,-0.14642788],[-0.20543787,-0.02313898,0.4434301],[0.30345842,0.43507323,0.3961148]],[[0.41857213,0.03291401,-0.04982631],[0.39721167,-0.4549522,-0.25229108],[0.13219374,0.01599317,-0.20964015]],[[-0.19138971,-0.5562733,0.10191873],[-0.30522764,-0.28711492,0.02635311],[-0.5178088,-0.27667516,-0.9339828]],[[0.04708187,-0.63850397,0.09435768],[0.40225583,0.36818382,-0.591937],[0.72178423,-0.43580616,-0.26423672]],[[0.42445174,-0.2948606,0.4204107],[-0.7016573,-0.0196372,-0.35594198],[-0.44545582,0.27891126,0.41212487]],[[0.18220676,-0.15999302,0.02349407],[-0.22300068,0.36395633,0.14243302],[-0.08783527,0.5232787,0.5116136]],[[-0.4409437,0.01500045,0.27826625],[-0.31548488,-0.01496076,0.756215],[1.3681808,0.24030703,1.1099819]],[[0.47879076,-0.22704063,0.09864086],[0.0906839,-0.5902771,0.4746413],[0.02359189,0.5880825,0.70466423]],[[-0.2831106,-0.553285,-0.2025805],[0.13895638,-0.01580045,0.21206136],[-0.0429121,-0.2805198,-0.89774865]],[[-0.01024787,0.24446632,-0.07427339],[0.06873447,0.79830146,-0.01754772],[-0.34639913,0.17727588,-0.6187367]],[[-0.3985501,-0.5582901,-0.09901798],[0.14809622,-0.32162562,-0.24273922],[-0.36072513,-0.04915321,0.55621576]],[[0.520657,-0.6290119,-0.511886],[-0.53913486,-0.13032655,-0.34356198],[0.11746313,-0.10830181,1.2155923]],[[-0.3861882,-0.25218472,1.5664345],[-0.26591232,-0.8023254,-0.07387373],[-0.17977619,-0.13703945,0.19805038]],[[-0.04422228,-0.11268124,-0.84227103],[0.12095789,0.3672742,0.14791931],[-0.06650543,-0.39817083,0.07906491]],[[0.45116326,0.10331813,0.46129173],[-0.09038929,-0.4143734,-0.15664862],[-0.03417044,-0.84518635,0.93472844]],[[-0.07875198,-0.52848977,-0.09408264],[-0.1546883,0.19135301,0.33277068],[0.5369275,-0.5815724,0.33255947]],[[0.23351099,0.04614713,-0.2278505],[-0.38846794,-0.14996085,-0.14518027],[-0.11302684,0.23047172,-0.3963586]],[[0.20255281,-0.5728744,0.1094334],[0.64490813,0.13325109,-0.06484083],[-0.18046571,0.00606175,-0.02868198]],[[-0.33780453,-0.519946,-0.1515094],[0.14754693,0.9673622,-0.20422839],[0.09925625,-0.47510636,0.6039543]],[[0.37444675,0.37335655,0.62916565],[0.02274364,-1.0827073,-0.39476797],[-1.0422531,0.6312106,-0.16899599]]],[[[0.54505044,0.06313766,0.22278349],[-0.05459564,-0.23442872,0.00642683],[-0.0242353,0.26947138,-0.5777366]],[[-0.46173933,0.27304682,0.14299178],[0.10067727,-0.09420596,0.67205626],[0.23239115,-0.00434604,-0.37713718]],[[0.43458435,-0.3960158,0.10097966],[0.01950496,-0.48995945,0.8601146],[0.23317067,0.1289149,0.2556428]],[[0.42475685,-0.07889713,-1.027863],[-0.24103962,-0.44156036,0.3711132],[0.17943983,0.38222617,-0.49541712]],[[0.19959566,0.58826274,-0.33257753],[-0.9809588,-0.9753793,0.4359616],[-0.29853073,-0.20512243,0.8106282]],[[-0.522098,-0.32221168,0.15096548],[-0.4803851,0.01739638,-0.9776427],[-0.14687884,-0.3145589,-1.0656238]],[[-0.496329,-0.336343,0.39376056],[-0.8976367,0.14431211,-0.2834163],[0.02875966,-0.1336305,0.46482208]],[[-0.38576865,0.49493983,-0.49697205],[-0.24261415,0.44870582,0.34841377],[-0.02987889,0.28083777,0.5453125]],[[0.50236446,0.13438849,-0.6015984],[-0.16080402,-0.2980112,-0.4445774],[-0.45644107,-0.86566603,-0.22113356]],[[-0.4641202,0.62547225,0.21495852],[-0.28904596,-0.10375981,-0.47293198],[0.02376593,-0.81658745,-0.07118057]],[[0.13470721,-0.20687862,0.22613019],[0.9449758,0.35880494,-0.05563617],[-0.04792639,-0.657074,0.9443995]],[[0.05889698,0.68343383,1.0835968],[0.6911963,0.02472651,-0.42660812],[-0.26823792,0.04061418,0.39100274]],[[0.53833884,0.14973532,-0.1956446],[0.20256127,-0.3207601,-0.74093467],[0.6262576,0.05408841,0.34071717]],[[-0.05404983,-0.21265382,-0.03942062],[0.09269874,0.7544555,-0.41009107],[0.08211708,0.42713103,-0.0510297]],[[0.21207727,-0.00249471,0.2138066],[0.39180586,-0.26228294,-0.5840922],[0.02427459,0.09007164,-0.47525528]],[[0.16622296,-0.303994,-0.23745161],[0.55012816,0.24026449,-0.5994787],[-0.74229044,-0.49895763,-0.09773553]],[[0.48017988,0.21070923,-0.02714303],[-0.9612525,-0.4467497,0.03979915],[0.64229316,-0.9955122,0.3350779]],[[0.10436501,-0.55164367,0.34031162],[0.69988513,0.68242115,0.87058973],[-0.39161766,0.59838283,0.52444106]],[[0.32325643,-0.4009043,-0.53736126],[-0.08419886,-0.53260404,0.56638896],[-0.3043536,-0.34061626,-1.0862356]],[[-0.56820416,0.26140693,-0.50260514],[-0.39730155,-0.85897267,-0.61912966],[0.46594945,-0.57578754,-1.3182168]],[[0.5954928,-1.2800578,-0.14096953],[0.6268274,-0.6604386,0.6723341],[0.2678256,-0.09041267,0.5827337]],[[0.5022392,-0.8376131,0.09423508],[0.48180246,0.00162914,-0.6784107],[0.30064383,-0.13887298,0.18274228]],[[-0.05058686,0.3343794,-0.5815195],[0.16568759,0.22418813,-0.61368084],[-0.347697,0.5287869,-0.34924686]],[[0.3345318,-0.16141708,-0.0097345],[-0.3382801,-0.46599987,-0.6119397],[0.2754112,0.40565425,-0.8231855]],[[-0.82392704,-0.20647933,-0.4059828],[-0.6048644,0.5320547,0.13551168],[-0.2773033,-0.04122376,-0.622574]],[[0.18887739,-0.15462513,0.08450235],[-0.17080422,0.01316508,0.22040755],[-0.12350362,0.25161147,-0.13571952]],[[0.11669485,-0.37020248,-0.12052312],[0.55600154,-0.1172609,-0.6822619],[-0.12766215,0.15997872,-1.3019489]],[[-0.79934883,-0.35567582,0.84652716],[0.09351522,0.00425196,0.21345493],[-0.37622985,-0.15135868,-0.31347466]],[[-0.19638605,0.1054018,0.29746345],[-1.0538466,-0.2768818,-0.7442921],[-0.12726338,0.6330705,-0.50099796]],[[-0.49789193,-0.2679067,0.01652879],[0.03601771,-0.59427786,-0.5417979],[-0.44360453,-0.40399703,0.27246115]],[[-0.41787943,-0.59405184,-0.4998994],[0.15973881,-0.44976854,-0.32924733],[-0.8241026,0.40453228,-0.07725592]],[[-0.04690216,0.08982463,0.09129123],[-0.31747627,0.19775505,-0.5170299],[-0.34183595,-0.6271211,-0.1010713]],[[0.10571209,-0.67237234,-0.61462617],[-0.27324647,0.33185625,-0.32363972],[-0.6167045,-0.11801071,0.7798323]],[[-0.28151956,0.06365441,0.16181031],[-0.31457135,-0.2388263,-0.309915],[-0.22212753,-0.06391969,0.73932105]],[[0.13092873,0.07137226,-0.24244848],[0.33037776,-0.11340963,0.47672874],[-0.37783563,-0.44495255,0.6023661]],[[-0.35690764,0.51946783,-0.0630715],[-0.2468234,0.16950205,0.19405332],[0.24252191,0.3856246,-0.90657896]],[[-0.20073657,-0.7245116,0.17483929],[-0.21367456,0.5349976,0.44367313],[-0.1925022,0.58773905,0.04357562]],[[0.26410303,0.27555576,-1.1395867],[-0.24280763,0.516668,-0.09662875],[-0.03760328,-0.01352204,-1.0153166]],[[-0.6448541,0.33933055,0.09169099],[0.16003315,0.70586616,0.05766336],[0.29737496,0.02872299,0.15181017]],[[-0.27010483,0.5978576,0.31861925],[-0.65112656,0.28317183,0.2296737],[0.35555568,0.2110155,-0.17580587]]],[[[5.14739215e-01,9.84843448e-02,4.97708052e-01],[-4.91448045e-01,2.00250953e-01,-8.76571611e-02],[-3.32626887e-02,5.90590715e-01,-2.74863899e-01]],[[-1.07134454e-01,-1.33113876e-01,1.84002906e-01],[-6.58107549e-02,4.74111289e-01,-6.47277594e-01],[1.85346618e-01,1.23010850e+00,-4.55717087e-01]],[[-1.12637002e-02,-1.07363880e+00,2.11843431e-01],[4.43151221e-02,-6.06059492e-01,-8.69852155e-02],[-3.99619013e-01,-1.42880097e-01,-1.69120207e-02]],[[1.02103904e-01,-2.18638092e-01,8.96941841e-01],[-2.27185547e-01,-8.75896215e-01,-5.26817024e-01],[8.20694983e-01,2.73704022e-01,-3.59598875e-01]],[[1.27554908e-01,4.31299508e-01,3.63624275e-01],[1.87263191e-01,3.73586118e-01,-3.91895138e-02],[-1.67534813e-01,8.75786692e-02,-3.19995016e-01]],[[7.93039143e-01,-2.63236701e-01,-2.91217119e-01],[1.03927411e-01,-4.48059529e-01,1.43091843e-01],[-1.99793026e-01,-3.88335675e-01,-3.35215718e-01]],[[5.33729307e-02,-4.21075165e-01,2.57383902e-02],[8.95077363e-02,2.11338133e-01,-4.18781906e-01],[2.94259265e-02,-1.28709108e-01,-6.47320151e-01]],[[-6.11285031e-01,6.84739277e-02,2.08858937e-01],[3.14405449e-02,2.43889183e-01,2.65468270e-01],[-8.06909144e-01,5.64635694e-01,7.26658881e-01]],[[-3.05611908e-01,-2.74789959e-01,-2.88455695e-01],[-7.57753193e-01,1.49531931e-01,-1.71740904e-01],[2.67313421e-01,7.37493694e-01,5.60134053e-01]],[[2.81372756e-01,-4.03145581e-01,4.07850415e-01],[-9.37030792e-01,-7.32606113e-01,-1.82403788e-01],[-1.10119045e+00,1.40845671e-01,-2.79631466e-02]],[[8.79092291e-02,-5.36112368e-01,6.63450509e-02],[-6.72245845e-02,1.16427667e-01,-2.79330701e-01],[-6.68389022e-01,-5.99919632e-02,-1.90919995e-01]],[[4.09275442e-01,-1.51686907e-01,-7.09288239e-01],[1.29868910e-01,2.16722369e-01,3.63843590e-01],[-5.19742906e-01,1.69278800e-01,-2.69748539e-01]],[[-4.80981261e-01,9.48987842e-01,-2.77210176e-01],[-5.69211781e-01,3.66270512e-01,8.07518840e-01],[8.53727236e-02,-4.49844748e-01,9.02973235e-01]],[[6.32199645e-01,-8.03974867e-01,1.01329915e-01],[6.84918106e-01,-5.47291338e-01,7.69650936e-01],[4.35089082e-01,-4.51323658e-01,-1.71315730e-01]],[[6.16311133e-01,8.10053200e-02,-7.36313045e-01],[-3.83778900e-01,-2.79058099e-01,6.46371841e-02],[5.50634444e-01,3.34743202e-01,8.61645103e-01]],[[2.85003066e-01,5.41467845e-01,-3.11872780e-01],[-2.74798006e-01,4.51730229e-02,-6.12303428e-02],[-2.57678628e-01,2.74088800e-01,-1.50231376e-01]],[[-6.37089729e-01,-1.72293901e-01,3.79472345e-01],[-9.73280966e-01,1.05753923e-02,2.43462071e-01],[-7.74298429e-01,-1.26892909e-01,-5.63523650e-01]],[[2.95820713e-01,-2.94604629e-01,-1.58465713e-01],[7.58699894e-01,-4.79326129e-01,3.88363987e-01],[3.49209160e-01,-1.08777834e-02,-1.19451866e-01]],[[-1.07125156e-01,1.57481432e-01,-1.43575892e-01],[3.72649461e-01,-2.31107071e-01,3.77192855e-01],[7.11607277e-01,-3.78950298e-01,-2.34257981e-01]],[[-2.53699481e-01,-5.93006089e-02,-3.81454080e-01],[-4.37144995e-01,-4.11466092e-01,-3.85562062e-01],[2.80150115e-01,-6.07238889e-01,-3.84318262e-01]],[[3.27038884e-01,6.85362399e-01,1.80672348e-01],[2.92882353e-01,5.08175850e-01,9.58271921e-02],[-4.66496378e-01,-4.85264361e-01,-3.98218989e-01]],[[-1.18209565e+00,-4.49762732e-01,-3.79815660e-02],[-2.63143241e-01,-2.59530962e-01,8.32399949e-02],[-5.39149582e-01,-2.96723336e-01,-2.46097878e-01]],[[-1.05405641e+00,-3.52585763e-01,3.33638787e-01],[-9.37692523e-02,-6.27870679e-01,2.32436150e-01],[7.82195553e-02,-3.97040159e-01,-1.97327554e-01]],[[2.84985900e-01,2.75893062e-01,-3.21524650e-01],[4.28187728e-01,1.21751195e-02,-4.31502223e-01],[1.34373760e+00,4.31376725e-01,-4.37468439e-01]],[[-3.35660398e-01,8.75975639e-02,-3.88811916e-01],[2.55607247e-01,3.52127224e-01,-3.94207202e-02],[-2.21801326e-01,-4.96460050e-01,5.63104898e-02]],[[1.77255332e-01,-3.70884806e-01,2.61265039e-01],[-5.56858361e-01,1.91723198e-01,-3.69676471e-01],[4.55554482e-03,4.00607318e-01,6.76715910e-01]],[[4.73667592e-01,-4.71083939e-01,-3.69436771e-01],[-4.49565314e-02,7.50092119e-02,-4.47880089e-01],[8.83639038e-01,-5.26341796e-01,2.42096752e-01]],[[9.99055028e-01,6.50135994e-01,3.33111547e-02],[2.78032690e-01,-3.09850108e-02,7.62458682e-01],[1.57974306e-02,-4.92774881e-04,2.05114290e-01]],[[-1.29198119e-01,1.80223718e-01,1.30931127e+00],[-3.97696406e-01,-2.86463741e-02,-3.61877352e-01],[-1.12493522e-01,-6.91157937e-01,2.23215044e-01]],[[1.18235976e-01,-3.77730548e-01,-1.70746177e-01],[3.43553334e-01,9.21184495e-02,-4.60638165e-01],[6.52800083e-01,1.01924442e-01,1.85674623e-01]],[[-6.02951795e-02,1.15644634e-01,-2.66999573e-01],[-3.83881122e-01,1.26372799e-01,1.68943048e-01],[1.76243722e-01,5.73390305e-01,-2.77880013e-01]],[[8.26062188e-02,1.15381107e-01,2.99564302e-01],[2.85029888e-01,-3.95614877e-02,4.37732518e-01],[3.76656353e-01,7.04483211e-01,-9.77421343e-01]],[[1.72533423e-01,-1.34261534e-01,-7.60438502e-01],[8.37362885e-01,7.20566586e-02,-5.66121578e-01],[4.99014139e-01,1.01213014e+00,2.20572710e-01]],[[-2.63551235e-01,-4.71970022e-01,2.22375661e-01],[1.88844483e-02,-4.70745385e-01,-3.60747635e-01],[4.42145586e-01,-6.58978298e-02,1.60799652e-01]],[[1.25209361e-01,8.30492198e-01,1.04961373e-01],[5.47793508e-01,2.20445484e-01,-6.23042956e-02],[-4.32765752e-01,2.81378150e-01,1.06894624e+00]],[[2.43996881e-04,2.19548404e-01,6.91509604e-01],[6.12699986e-01,1.56190600e-02,5.00359535e-01],[-7.03929007e-01,1.88660517e-01,-2.53019005e-01]],[[4.28908110e-01,6.04334712e-01,-5.74925900e-01],[6.17909491e-01,-1.65339276e-01,-5.90895295e-01],[4.41041291e-01,8.98088932e-01,7.72527158e-02]],[[1.01695962e-01,6.19430065e-01,4.42078352e-01],[-1.04989243e+00,-7.21205533e-01,2.62755692e-01],[5.29527962e-01,-5.70876837e-01,-5.08792400e-01]],[[3.45132232e-01,-3.78779054e-01,9.73787457e-02],[5.09327412e-01,-8.94937143e-02,-1.90835431e-01],[-4.85136002e-01,-9.34030712e-02,-1.35527760e-01]],[[6.05651855e-01,-9.33066607e-02,-1.38225496e-01],[5.44521846e-02,-1.87229082e-01,-2.52316236e-01],[-1.10451114e+00,4.70607191e-01,4.32762578e-02]]],[[[-0.8101431,0.3279559,0.39059845],[-0.19289587,0.39215794,-0.06672611],[0.0298598,0.21418905,-0.369881]],[[0.2680835,-0.2848796,-0.51260614],[-0.32425603,0.09963918,0.30995932],[0.08964834,-0.09938525,0.25922617]],[[0.5882312,0.09669378,-0.43919244],[0.49495882,0.11156388,0.15787177],[-0.15745023,-0.4743416,0.32790047]],[[-0.38362843,0.02971965,0.96884865],[-0.40987,-0.8511239,0.01953417],[0.37746125,0.43444335,-0.3558827]],[[-0.25614432,-0.10755939,-0.14184141],[-0.2602315,0.14591476,-0.27271676],[0.5643232,0.25144327,0.3099341]],[[0.0240221,0.45811132,-0.09184992],[0.07459638,-0.4925686,0.56200254],[0.09083465,-0.00658232,-0.47704974]],[[-0.03239417,0.23199749,-0.35585308],[0.36486688,0.10207003,-0.30457878],[0.80004644,-0.16704899,0.3922327]],[[0.13408698,-0.61583394,0.10563689],[-0.32791963,0.42671862,0.32212418],[0.42239693,0.29117796,-0.26661807]],[[0.30516723,-0.4856533,0.1123367],[-0.14576,0.08800703,0.15298422],[1.0074881,-0.7303461,-0.23493198]],[[0.04682422,-0.4231313,0.29746133],[-0.01348817,-0.49509752,0.6414966],[1.0951209,-0.7036338,0.05842301]],[[-0.2671636,0.38517043,0.40908092],[-0.5617559,-0.60819054,-0.5800485],[-0.8174761,0.1494072,0.01847975]],[[0.31252673,0.3791275,0.26623496],[0.08849425,0.2730738,-0.00304173],[-0.01812366,0.37445858,-0.1454997]],[[-0.3936209,-0.25544325,0.7436805],[-0.9093014,-0.4140691,0.06178097],[-0.00460994,0.93700016,-0.31226936]],[[-0.11906558,0.30226249,0.31956837],[-0.46909183,-0.38765815,-0.00592155],[-0.11135869,0.34818125,-0.16091654]],[[-0.2769388,0.23114619,0.1339046],[0.05058991,0.5088403,0.09146971],[-0.6485632,0.10408214,-0.42338264]],[[0.20768212,-0.24705517,0.1276406],[0.39122427,0.3137402,-0.51287115],[0.77503943,0.07081428,0.32893017]],[[0.44742626,-0.2980308,-0.40019965],[0.501753,-0.82198304,0.7787894],[1.8071907,-0.07869298,0.31380975]],[[0.32534462,-0.14606951,-0.23269407],[-1.0404024,-0.18334678,-0.6269427],[-0.02324093,-0.18021065,0.1950117]],[[-0.5920349,-0.32808432,0.2849508],[-0.13918753,0.77855194,0.28807575],[-0.80475324,0.18930468,-0.58540714]],[[-0.41947576,0.6361672,0.4566382],[0.67463416,0.6923064,0.24687187],[0.93962526,-0.11451015,0.22631332]],[[0.22332479,-0.4817113,0.21817096],[-0.08274927,0.57170606,0.3497883],[0.08701351,0.08710326,-0.32870504]],[[1.9164683,0.22258425,0.12120105],[1.1687198,0.3669312,-0.3042561],[0.5296523,-0.5382808,-0.07037798]],[[0.09428672,0.48823085,0.17680804],[-0.43709227,0.3575737,0.08812699],[-0.68230885,-0.1612658,-0.49919853]],[[-0.32609147,-0.12651356,0.24824665],[0.03482464,0.1382112,0.9432988],[-0.60463,0.2949349,0.6006497]],[[0.9946699,0.35909402,0.10866528],[0.4240743,0.20828293,-0.02930264],[0.673516,0.10993784,-0.29867828]],[[0.50079435,0.31258082,-0.17053255],[0.17410086,-0.09560723,0.33045977],[0.8752725,-0.49804023,-1.2738874]],[[-0.24588016,-0.04914834,0.2985518],[-0.15332797,0.5548059,0.44212583],[0.48984408,0.5022282,-0.7022046]],[[-0.6009981,-0.19719878,0.02504954],[-0.2061051,0.050417,-0.1326452],[0.5509183,-1.0417395,-0.05695409]],[[-0.3240411,-0.515743,0.10114995],[-0.78738886,0.15953845,0.467143],[0.6710604,-0.27381343,-0.3934653]],[[-0.40161481,0.2978252,-0.11059361],[-0.01907502,-0.01331264,0.5089138],[-0.528185,0.0940565,-0.35546696]],[[0.10854858,-0.44137317,0.04461371],[-0.04532302,-0.31582195,0.05971253],[0.20855725,0.10287195,0.02796701]],[[0.7824534,-0.18967742,0.25367814],[0.07369719,0.8384841,0.00442186],[1.1385165,0.32377037,0.46263528]],[[-0.2413391,-0.26044703,0.3889577],[-0.7506472,-0.49663684,0.13121845],[0.02507504,-0.69322795,0.6897564]],[[0.29535425,-0.73290324,0.25481105],[0.0898473,0.2289534,-0.42375085],[0.5879335,-0.37385616,0.14206368]],[[-0.55103034,0.26711372,-0.22088504],[-0.06970738,0.0946769,-0.14102015],[0.03506633,0.14571959,0.5730731]],[[-0.02080215,-0.31436053,-0.424818],[0.18117706,-0.18058014,0.24187115],[0.08888435,-0.43277875,0.15223528]],[[0.2238484,-0.24805473,0.55660707],[0.09198701,0.58849806,-0.1681436],[0.20247422,-0.7925535,-0.52929103]],[[0.96885157,0.3352621,0.27983502],[0.7898119,0.28675386,0.12918003],[0.6163763,-0.2833608,0.6433274]],[[-0.6546561,0.15901837,0.02580163],[-0.49120402,0.30702564,-0.19609654],[1.1505394,0.11503622,0.14556788]],[[-0.28109398,-0.19161429,0.23201783],[0.3082124,-0.19245586,0.26650733],[0.39469114,0.74844396,-0.6274789]]],[[[-0.18999404,-0.49436232,-0.41585895],[-0.4136204,0.16059595,-0.07630116],[-0.10739139,0.57779443,-0.09927442]],[[-0.52900165,0.07013902,-0.40277928],[0.08094251,0.69390166,-0.6355976],[-0.6562533,0.47383606,1.0126938]],[[0.14108492,-0.340561,-0.1390835],[-0.33752692,-0.04566177,0.02487222],[-0.82567906,-0.28582865,0.20937751]],[[-1.0338702,0.5460475,-0.94392174],[0.3222821,-0.6847733,-0.9102191],[-0.91857696,-0.43266273,-0.4757535]],[[-0.09298421,0.20951968,0.45192343],[0.2057892,0.53370166,-0.43246284],[-0.6258491,0.38354665,0.9404781]],[[0.22865859,-0.50237507,-0.6916211],[0.1079336,-0.3837447,-0.20793061],[-0.11268341,0.1039087,0.15908764]],[[-1.3356497,0.53753597,0.30754703],[0.1674671,0.48114917,-0.00298564],[0.09707154,-0.568811,-0.0374628]],[[-0.43999434,0.15517795,-0.07959975],[-0.08257618,0.5033245,0.15120228],[-0.39095142,0.15632538,-0.23736402]],[[-0.19979867,-0.31316292,-0.14163776],[0.8260102,-0.22897863,0.30145365],[-0.7117863,0.07795291,0.14089498]],[[0.03842278,0.02349739,0.68339145],[0.74681133,-0.20017047,0.1799778],[-0.13197888,-0.10872489,-0.19401392]],[[-0.49861172,0.24963047,-0.24413761],[-0.13894944,-0.18605398,-0.30317402],[0.01036005,0.40950763,-0.04256351]],[[0.24692215,0.10504568,0.25422126],[-0.611212,0.40087134,-0.35335398],[0.5614595,-0.528545,-0.16069177]],[[0.28566548,-0.51228523,0.5811581],[0.3232082,-0.39128742,0.67870027],[0.17387195,0.09956297,0.0844956]],[[0.8059177,0.28429034,-0.13161749],[-0.38895354,0.9187714,-0.9910059],[0.71331006,0.00259881,-0.3573619]],[[0.07242475,0.64134026,0.0460817],[-0.6492072,-0.5630052,-0.5993346],[0.22796997,-0.88828427,0.75054]],[[-0.04647578,-0.1782329,-0.13124715],[-0.42531645,-0.3201888,0.83210987],[-0.25698692,0.281343,0.89975625]],[[-0.36921948,0.21394446,0.33847106],[-0.02221593,-0.05638243,0.14965755],[0.14055394,0.03834089,-0.7645311]],[[1.0339894,0.19503833,-0.1711705],[0.00348179,-0.9042495,-0.6954751],[-0.602836,0.92478704,-0.05593033]],[[-0.05154748,-0.21425901,0.63844466],[0.4930289,0.44770816,-0.08950752],[-0.29259688,0.63402253,-0.59291416]],[[0.05124221,0.2117605,0.78684807],[-0.08361708,-0.48390073,0.2656175],[-0.4092898,0.40131897,-0.14245646]],[[-0.36715713,-0.43973532,0.5915333],[-0.8145573,0.316609,0.31188536],[-0.08231583,0.46170563,-0.19892557]],[[-0.0295262,-0.7007824,-0.07789707],[0.07538062,-0.6721383,-0.07452423],[0.15898523,-0.29519683,-0.18604337]],[[0.18955825,-0.4469822,-0.8225622],[0.09468377,0.6652992,-1.184432],[0.67283607,0.24499351,-0.11294249]],[[0.23916851,0.5446232,0.02546978],[0.40270114,0.7470581,-0.4247755],[0.5051737,0.5499192,-0.19096345]],[[0.4909775,-0.3312993,-0.05782181],[-0.01436506,-0.01534311,-1.1653095],[0.01961089,0.45918074,-0.07609367]],[[0.43969244,-0.36234805,-0.31005964],[-0.8239056,0.46085167,0.10867459],[0.80731785,-0.42461357,0.09026389]],[[0.38376024,-0.14588223,0.02863852],[-0.28138113,-0.38674104,-0.01629068],[-0.45401514,0.01678695,-0.7410088]],[[0.6328772,0.20791997,-0.27221307],[0.11543097,0.1286858,0.0523332],[0.15430124,-0.12827921,-0.28011936]],[[0.29685974,-0.14617102,1.3426648],[-0.4328571,-0.08411434,0.4899358],[0.1541874,0.23239136,0.18497089]],[[0.5181786,-0.29121253,-0.58627725],[-0.28731263,-0.44766775,0.9425254],[0.8049196,0.5028787,0.57156426]],[[-0.640312,0.89342576,0.39211124],[-0.16916811,0.4071409,0.22639441],[-0.09319393,-0.1371233,0.21859947]],[[-0.309786,-0.47064602,0.37283653],[-0.28801906,0.04721519,-0.13329835],[0.5176684,0.23306894,-0.5171073]],[[0.04641604,0.10223396,-0.03821931],[-0.46192548,-0.2893471,-0.3709344],[-0.2613228,0.8713159,0.35484993]],[[0.40013424,0.93225354,0.35109314],[-0.57778364,0.0399649,0.5306405],[0.0572154,-0.7130913,-0.09089775]],[[0.14242293,0.55961156,0.59437317],[-0.32658923,0.30700746,0.6250057],[0.01845361,0.1777374,0.5179166]],[[-0.84775543,-0.98741496,-0.24090482],[-0.549238,-0.04560504,-0.00346205],[0.36963966,-0.8625837,-0.16897236]],[[0.4094118,0.04446973,0.17706834],[0.18244433,-0.59304935,-0.39105758],[0.81436205,0.1561393,-0.48074114]],[[0.38872692,0.06579158,0.1798743],[0.49293086,0.1953109,-0.4351687],[0.59288675,-0.79793006,0.39442697]],[[0.13734439,0.34991947,-0.52880454],[1.0313668,-0.48327616,0.03729618],[0.16938858,0.11157174,0.11954939]],[[-0.03180979,-0.6186703,0.1446723],[0.02697702,0.75109845,0.0503749],[0.14646658,-0.4179329,0.23190567]]],[[[3.10275495e-01,-5.52223504e-01,-3.83819759e-01],[-5.72740734e-01,9.20756042e-01,-2.14119300e-01],[6.63904369e-01,-1.53320849e-01,-4.07838136e-01]],[[-2.26428092e-01,8.71291995e-01,-1.71975508e-01],[-3.29004049e-01,1.74106434e-01,-1.20015226e-01],[-1.01453386e-01,6.66981414e-02,-1.07918358e+00]],[[-1.95393786e-01,-1.37005672e-01,1.07274652e+00],[-2.57443756e-01,-6.60576880e-01,6.52239203e-01],[-5.82551360e-01,-9.36984599e-01,2.14500770e-01]],[[3.62919629e-01,5.72669685e-01,-1.40886515e-01],[-3.32256228e-01,5.53472266e-02,-6.16303444e-01],[-5.36557913e-01,-1.16960369e-01,-2.43226647e-01]],[[3.63641279e-03,4.73212540e-01,-4.41289008e-01],[-9.35790300e-01,5.84752895e-02,-9.10201445e-02],[3.27332109e-01,-2.91153520e-01,1.06752105e-01]],[[-6.72191560e-01,-4.06713307e-01,4.07215983e-01],[-1.12482361e-01,1.89083219e-02,-5.45604646e-01],[1.02411294e+00,-4.52248082e-02,3.50841880e-01]],[[-4.96706456e-01,-3.57900262e-01,7.19209790e-01],[9.43992753e-03,9.00581479e-02,4.90437299e-01],[-1.09556961e+00,2.95249909e-01,1.30277619e-01]],[[-1.93205163e-01,7.75878370e-01,-4.29892950e-02],[8.06224644e-01,2.08203882e-01,-1.84998006e-01],[5.60755990e-02,3.12337816e-01,2.40286320e-01]],[[9.41365898e-01,-6.51256084e-01,1.71632603e-01],[6.12624824e-01,3.24635386e-01,-4.85981345e-01],[2.33180895e-01,2.07255080e-01,6.35011569e-02]],[[2.89699525e-01,2.58025289e-01,1.98743746e-01],[-8.08287263e-02,5.52214026e-01,-1.82315946e-01],[5.42051643e-02,7.75027215e-01,-6.60968244e-01]],[[-8.90831277e-02,1.27120148e-02,7.53386319e-02],[1.49485603e-01,1.09917931e-01,-7.61661291e-01],[-6.61211610e-01,1.06289811e-01,-1.12485506e-01]],[[1.00467421e-01,5.56238964e-02,4.60321009e-01],[7.63536036e-01,-1.25631183e-01,4.29507554e-01],[-7.06042111e-01,1.40193820e-01,8.04834247e-01]],[[-7.10954070e-02,9.52523053e-02,1.62195444e-01],[-2.14177087e-01,1.16625242e-03,3.23028386e-01],[-2.09017787e-02,3.27116758e-01,8.53504896e-01]],[[1.96538717e-02,4.00560588e-01,-2.27262124e-01],[-5.02823852e-02,-3.25094074e-01,-6.62617851e-03],[8.83750141e-01,-3.16607177e-01,-1.37603149e-01]],[[7.83348680e-02,-1.67556182e-01,-1.57106742e-02],[3.07808518e-01,1.84352487e-01,3.37245502e-02],[-8.08118433e-02,-1.39977777e+00,-2.40164161e-01]],[[1.32507920e-01,3.13155919e-01,-2.06185970e-03],[4.50115204e-01,9.05004382e-01,-1.38012692e-01],[1.72431394e-01,9.19583917e-01,-2.26230115e-01]],[[-4.26409155e-01,4.14139271e-01,2.58496881e-01],[-2.89303780e-01,1.26992390e-01,6.60704821e-02],[-5.34491897e-01,-1.46399125e-01,6.27809912e-02]],[[-2.76584923e-01,4.67860788e-01,4.98065770e-01],[-2.50178099e-01,5.60118966e-02,-1.68660358e-01],[9.14591074e-01,-3.22009772e-01,4.49037850e-01]],[[-2.50943094e-01,3.27890813e-01,2.03068145e-02],[2.07182169e-01,-5.06734252e-01,-4.10704046e-01],[9.98277545e-01,-4.61038589e-01,-5.18014550e-01]],[[2.02222139e-01,-3.29375267e-01,-3.83057147e-01],[-5.78688502e-01,2.69849271e-01,-9.27322358e-03],[-4.34845462e-02,1.73141193e-02,-2.99415499e-01]],[[-2.25606546e-01,-1.72097951e-01,5.66309690e-01],[-7.99088359e-01,8.42990518e-01,-9.61773992e-01],[1.87199488e-02,2.43364424e-01,-6.30379856e-01]],[[-2.76593029e-01,-1.13192499e+00,-8.87396857e-02],[-1.36144027e-01,5.55819094e-01,1.09628029e-01],[4.95943099e-01,-3.27105254e-01,-1.80593744e-01]],[[-3.04216117e-01,5.79132795e-01,-1.08824480e+00],[2.37478420e-01,4.96306479e-01,-4.79292870e-02],[3.18686038e-01,-2.06268981e-01,5.91170713e-02]],[[-5.30697286e-01,2.17772186e-01,-6.28271997e-01],[3.05316508e-01,5.52852392e-01,7.30934218e-02],[-7.08185555e-03,5.46204448e-01,1.93391532e-01]],[[-2.86772072e-01,2.19687790e-01,-4.94591981e-01],[-6.15094364e-01,2.16443732e-01,-5.64521968e-01],[3.18548828e-01,2.57919639e-01,-3.82904977e-01]],[[6.29033446e-01,-8.97753835e-02,7.51786709e-01],[9.51728880e-01,-6.65201694e-02,2.75498658e-01],[-4.59785014e-02,1.04842648e-01,2.87357002e-01]],[[-1.17363729e-01,-2.26562366e-01,-1.01567633e-01],[-2.09377445e-02,2.64396906e-01,-7.19875097e-01],[3.30785066e-02,1.56589493e-01,3.30353796e-01]],[[5.97101867e-01,-3.60954046e-01,-4.24568921e-01],[7.05505192e-01,4.41790909e-01,-7.65332997e-01],[1.19155788e+00,2.42531160e-03,-7.98552513e-01]],[[-1.06575206e-01,4.28394526e-01,7.55856276e-01],[4.41851109e-01,2.36713722e-01,-2.35162690e-01],[-4.15345550e-01,7.22396821e-02,-1.04331143e-01]],[[-2.34367582e-03,2.16020327e-02,1.76663592e-01],[-2.54386336e-01,-4.54344600e-01,2.76402265e-01],[1.08333759e-01,1.81878619e-02,6.18066132e-01]],[[-7.96682358e-01,3.14191788e-01,3.39028209e-01],[-4.30140972e-01,-3.20247918e-01,-1.77085102e-01],[-3.37108195e-01,1.52811706e-01,1.10811673e-01]],[[-4.97042537e-01,6.37992546e-02,-7.68769812e-03],[-1.03212357e+00,-2.01419979e-01,1.43020129e+00],[-1.56244999e-02,-5.55025339e-01,5.42888105e-01]],[[-2.76094973e-01,6.74903274e-01,1.10285029e-01],[1.40092716e-01,-5.98771214e-01,-1.64236099e-01],[-2.85122767e-02,-3.27512234e-01,-2.61133701e-01]],[[2.35759333e-01,5.66884756e-01,3.83910209e-01],[5.03445506e-01,1.58623204e-01,5.01325876e-02],[1.07192528e+00,1.70791328e-01,1.75949827e-01]],[[4.82727408e-01,-2.92998403e-01,-4.21942204e-01],[4.98494804e-01,6.03993498e-02,-3.38764101e-01],[5.64109325e-01,-1.40059009e-01,8.28722775e-01]],[[1.19766124e-01,4.77747023e-01,1.03019089e-01],[-1.33853948e+00,-4.33112592e-01,-3.15009683e-01],[-2.80038744e-01,1.24789202e+00,-1.09132931e-01]],[[3.23797345e-01,-5.56914389e-01,1.63606051e-02],[-8.45446944e-01,3.59397888e-01,1.93488777e-01],[-3.47197175e-01,5.59676528e-01,1.13537312e-02]],[[-9.70927179e-01,-3.86291087e-01,-5.56100905e-01],[-1.75425470e-01,-5.60993314e-01,7.23418772e-01],[-2.88110018e-01,-3.11309516e-01,5.90861022e-01]],[[-4.07853663e-01,1.32868603e-01,-2.71534622e-01],[-2.92170107e-01,3.06447178e-01,3.65591764e-01],[1.17683046e-01,-5.53702235e-01,-1.68037206e-01]],[[4.39998299e-01,-4.46450084e-01,-6.56611025e-01],[-1.43709376e-01,5.15248656e-01,-3.74590069e-01],[-2.44524196e-01,-1.10418178e-01,-3.28283340e-01]]],[[[6.44600570e-01,6.88024044e-01,8.72623622e-01],[5.90637505e-01,2.89538354e-02,-2.64814973e-01],[-3.85964721e-01,7.34586000e-01,-3.48968878e-02]],[[4.10728037e-01,-7.76517212e-01,-7.82595575e-02],[-2.86166042e-01,1.39974225e-02,-2.51363546e-01],[1.88280717e-01,-3.99537086e-01,-4.41196263e-01]],[[-3.61807495e-01,6.90333486e-01,-3.74496937e-01],[1.53741017e-01,1.73527896e-01,-3.78482968e-01],[3.22235286e-01,-5.57736531e-02,1.75487578e-01]],[[-4.66071874e-01,-6.48806632e-01,-6.72773495e-02],[8.02933797e-02,-5.53234279e-01,1.49672240e-01],[3.42893124e-01,-6.64354742e-01,8.58638942e-01]],[[3.95762511e-02,4.97039616e-01,1.16544998e+00],[8.82128440e-03,-4.37112637e-02,5.11234760e-01],[7.32115924e-01,-9.63053942e-01,7.15355515e-01]],[[-2.17259992e-02,-7.12436676e-01,-3.50497872e-01],[-6.66851759e-01,3.33570004e-01,7.46271133e-01],[3.14717203e-01,1.96911946e-01,-1.43802866e-01]],[[-6.60681129e-01,2.88832366e-01,1.50650114e-01],[-9.85104084e-01,-7.26257443e-01,-7.40843490e-02],[-2.24785641e-01,3.89488161e-01,-2.75853090e-05]],[[3.54931295e-01,-3.06973547e-01,-7.04825461e-01],[3.40434313e-01,4.29388821e-01,-2.11518630e-01],[-4.62884665e-01,-3.94172445e-02,-1.91448897e-01]],[[-2.78455645e-01,-1.25632614e-01,-1.44550502e-01],[-4.52981651e-01,-2.44263321e-01,-5.70006251e-01],[3.77388410e-02,-5.77187181e-01,-4.61722940e-01]],[[-1.70105308e-01,-5.34535646e-01,-1.90403208e-01],[1.78921863e-01,-3.75809520e-01,2.62547135e-01],[-2.43967697e-01,-8.80090773e-01,-2.32538600e-02]],[[-7.45911300e-02,2.11084977e-01,1.15760893e-01],[-3.47969353e-01,-5.09833619e-02,-4.23632056e-01],[-1.00453770e+00,-8.88114810e-01,1.94713637e-01]],[[-9.37222898e-01,-9.81550217e-02,2.43418738e-02],[5.96169904e-02,4.75404471e-01,-2.00271010e-01],[-5.11772484e-02,6.42483830e-02,-3.35199893e-01]],[[9.31771547e-02,-6.18956566e-01,3.37251648e-02],[5.35905242e-01,3.02518874e-01,3.35725158e-01],[-2.76956797e-01,6.77177832e-02,-7.45113799e-03]],[[5.99865377e-01,1.77685261e-01,-2.09851816e-01],[-2.12052032e-01,-6.21286668e-02,1.78918034e-01],[-2.15315893e-02,-9.45586741e-01,-8.59345436e-01]],[[2.75222123e-01,1.13389947e-01,-1.82794601e-01],[7.48685181e-01,-9.98245001e-01,5.63999295e-01],[1.76941767e-01,4.75710779e-01,-1.86844409e-01]],[[-1.04740784e-01,-3.54297101e-01,-1.09937415e-01],[2.39559159e-01,5.62990963e-01,7.43976891e-01],[-9.72216189e-01,-2.66469061e-01,-6.30580187e-01]],[[-7.60726109e-02,-5.20403087e-01,-4.91764277e-01],[-5.60913384e-01,1.86375231e-01,5.01551665e-02],[4.59173262e-01,-8.26557159e-01,3.72274190e-01]],[[2.21721129e-04,-7.27084756e-01,-1.87376916e-01],[-6.76777840e-01,7.81857222e-03,-5.18955469e-01],[6.47711694e-01,-4.31552827e-01,7.76285231e-01]],[[-1.02431349e-01,-2.27445647e-01,-9.99822840e-02],[3.70848864e-01,3.83581519e-01,3.64306927e-01],[-1.80913191e-02,6.96671844e-01,1.89029817e-02]],[[-5.28465390e-01,5.88720068e-02,-4.68987942e-01],[-1.39515430e-01,-9.87807155e-01,3.07420343e-01],[-3.90065074e-01,-2.01266930e-02,-5.33869684e-01]],[[5.44465125e-01,-5.88307738e-01,-8.34601104e-01],[8.84807229e-01,-2.02410311e-01,3.32762539e-01],[-4.14254189e-01,5.07227421e-01,-2.83672605e-02]],[[-2.75789320e-01,1.19375944e+00,1.80315107e-01],[-5.82573079e-02,-1.31733581e-01,-9.18976292e-02],[-3.77185158e-02,-2.81170458e-01,1.58878297e-01]],[[1.12290256e-01,-1.02488445e-02,-9.44588661e-01],[-2.87104011e-01,-1.14279762e-01,-2.92021275e-01],[-1.25716984e-01,3.76255572e-01,1.72732875e-01]],[[2.48868197e-01,-2.82911658e-01,1.18230708e-01],[-2.00658932e-01,-1.97524875e-01,8.67727622e-02],[1.77455068e-01,-2.64597625e-01,-7.34176755e-01]],[[-6.51240468e-01,5.60744584e-01,-1.80046633e-01],[8.48016217e-02,-8.00222814e-01,-2.08811253e-01],[-1.27979487e-01,-3.78052771e-01,-8.05909336e-01]],[[-4.28056926e-01,1.17645472e-01,-1.07807159e-01],[1.32717893e-01,1.31433144e-01,4.68408614e-01],[1.34232223e-01,5.00123322e-01,-4.02662069e-01]],[[2.18070626e-01,6.83311224e-02,2.12358743e-01],[-2.12076887e-01,6.28899932e-01,-1.13454998e-01],[2.74402142e-01,3.42366755e-01,-8.95638019e-02]],[[5.11254907e-01,-2.16044281e-02,1.04352869e-01],[-5.56503117e-01,-3.38925838e-01,3.28710020e-01],[8.75169262e-02,2.88837880e-01,-7.80353665e-01]],[[-9.92249191e-01,1.38722509e-01,1.25559008e+00],[-5.10116398e-01,-9.75808918e-01,6.40410602e-01],[-2.21857876e-01,-5.96699953e-01,3.08446109e-01]],[[2.08017111e-01,4.00434941e-01,3.09837580e-01],[-5.32811165e-01,1.05656385e-01,2.55661398e-01],[6.45545542e-01,-8.88111770e-01,6.94073200e-01]],[[1.92958009e+00,3.86938006e-01,-3.16837072e-01],[-3.03521790e-02,-2.66413778e-01,2.92331159e-01],[-4.74729151e-01,7.99483180e-01,-2.83673584e-01]],[[-4.42292541e-01,-3.06832463e-01,2.87216604e-01],[2.42055103e-01,2.81266302e-01,-8.59103873e-02],[2.81977773e-01,8.37414861e-01,-3.36770773e-01]],[[8.96714568e-01,6.57734498e-02,-2.81048089e-01],[1.01940203e+00,1.02099848e+00,4.25000876e-01],[3.77665699e-01,3.78984064e-02,3.30989450e-01]],[[1.94438353e-01,1.19763933e-01,-2.85165429e-01],[4.14278507e-01,-1.50242373e-01,-1.24369055e-01],[4.11807857e-02,9.63321269e-01,5.13004959e-01]],[[1.84349343e-01,8.37165564e-02,1.31840520e-02],[-2.32045859e-01,5.80610931e-01,-1.17981821e-01],[6.37254119e-02,9.23691988e-02,3.28668296e-01]],[[4.52654898e-01,-4.59401459e-02,3.13178122e-01],[1.92268193e-01,1.24721147e-01,7.80028850e-02],[-1.19210124e-01,-1.52552664e-01,-9.86392796e-02]],[[-1.11807168e+00,3.34044062e-02,4.51678708e-02],[-3.46055895e-01,1.82206318e-01,-3.64342511e-01],[7.02296719e-02,-1.49398908e-01,7.42349148e-01]],[[2.68987238e-01,-3.94012630e-01,-1.96263865e-01],[2.34697938e-01,8.17601144e-01,-6.87125742e-01],[-2.81241089e-01,-6.41349852e-01,4.40809876e-01]],[[2.57554501e-01,6.69199407e-01,4.24301893e-01],[-1.90315157e-01,1.22844435e-01,-9.34502661e-01],[3.70961845e-01,-7.51553535e-01,-3.27620246e-02]],[[1.88399330e-01,-4.45576698e-01,-7.12960601e-01],[3.19687724e-01,-1.65364832e-01,4.09429282e-01],[4.45054889e-01,4.52162623e-01,7.27862775e-01]]],[[[1.97055280e-01,9.35590416e-02,1.58746347e-01],[4.73814458e-01,-3.06046247e-01,-1.37578115e-01],[6.35975152e-02,8.43027353e-01,3.22980076e-01]],[[1.61027893e-01,2.13031203e-01,2.55094349e-01],[-6.19701982e-01,1.93184633e-02,2.73784161e-01],[1.11976855e-01,-6.26313329e-01,4.17438187e-02]],[[-3.36531132e-01,-3.71948242e-01,1.29062399e-01],[-1.81475714e-01,-1.54051498e-01,-4.60166596e-02],[4.23666656e-01,-1.48328349e-01,-6.26089036e-01]],[[1.75300658e-01,4.60570343e-02,3.90852429e-02],[8.78492713e-01,-3.04720849e-01,1.73199754e-02],[1.82644993e-01,1.04113519e+00,-5.73525112e-03]],[[1.34903759e-01,-6.66868567e-01,8.70727599e-01],[4.46637779e-01,-1.20493062e-01,-4.02668864e-02],[-3.55476350e-01,-4.69830295e-04,8.58914137e-01]],[[-3.72074246e-01,-6.44090712e-01,3.42405260e-01],[-2.34161898e-01,-1.98866263e-01,4.88862008e-01],[-1.46181619e+00,-2.51774669e-01,-8.65846694e-01]],[[2.01923355e-01,7.16718137e-02,-8.02654505e-01],[-5.61469913e-01,-3.12694252e-01,-8.78402144e-02],[-1.16793595e-01,2.71918386e-01,-4.15194035e-01]],[[-3.83682430e-01,-5.86358190e-01,6.36588037e-01],[-2.15663426e-02,-3.32462847e-01,-1.19131315e+00],[-5.94843686e-01,-2.07336098e-01,-1.03686489e-01]],[[-3.87110084e-01,3.62611532e-01,-3.52560580e-01],[9.00474906e-01,1.99155018e-01,2.48881057e-01],[-4.70561028e-01,3.45934868e-01,5.53699955e-02]],[[1.70228899e-01,2.84563094e-01,-1.14743821e-01],[-3.71811688e-01,-7.45146632e-01,-2.49964550e-01],[-3.82268041e-01,9.61497605e-01,-5.38663507e-01]],[[1.18512131e-01,2.88836718e-01,2.11785473e-02],[-6.93595409e-01,-1.97726265e-02,1.65224507e-01],[3.21508259e-01,6.74928963e-01,1.59998909e-01]],[[1.73493326e-01,-4.97969270e-01,1.21533573e+00],[-8.77401114e-01,-3.51906598e-01,2.20907375e-01],[-2.09508881e-01,3.71495634e-02,-3.22373919e-02]],[[-6.29037805e-03,-8.14466625e-02,-2.71258712e-01],[5.57954125e-02,6.62260771e-01,1.59581572e-01],[1.46318272e-01,-5.98178923e-01,-6.96873307e-01]],[[1.00998402e+00,-4.56672907e-01,-2.09226906e-01],[7.88278937e-01,-3.24927568e-01,-1.66407168e-01],[3.72142613e-01,4.63217229e-01,-4.95418519e-01]],[[-4.76873875e-01,-4.60096955e-01,3.48628491e-01],[7.09173799e-01,-2.69241065e-01,4.60126903e-03],[3.39836925e-01,3.04376811e-01,6.65555000e-01]],[[5.36816955e-01,-1.74501222e-02,9.50503424e-02],[2.69744217e-01,6.10933065e-01,-9.78336483e-02],[-5.90622187e-01,5.55075228e-01,-4.15553851e-03]],[[-5.65578230e-02,3.84629786e-01,-1.15783578e-02],[8.44008386e-01,-2.18135804e-01,5.06108701e-01],[-3.35929096e-01,1.61990169e-02,-4.35443223e-01]],[[7.56670833e-01,4.20634300e-01,-1.82132199e-01],[1.80374801e-01,1.13009661e-02,1.09248541e-01],[6.96227849e-01,4.24765944e-01,6.34260714e-01]],[[-1.96646988e-01,-3.37215126e-01,3.45891505e-01],[-5.64359307e-01,4.38031256e-01,5.77399135e-01],[-2.75861304e-02,-4.19770032e-02,-7.21604645e-01]],[[-1.91658303e-01,6.28854573e-01,-9.55915272e-01],[-5.36837399e-01,-3.66758972e-01,9.90485430e-01],[-3.87511671e-01,2.64594525e-01,-9.05954480e-01]],[[2.98566610e-01,-2.14244813e-01,3.27415437e-01],[-9.74128321e-02,-5.16317822e-02,2.91273236e-01],[4.97149676e-01,1.63257748e-01,-1.05028324e-01]],[[-4.79562610e-01,-2.00110480e-01,2.64231414e-01],[-3.28373522e-01,4.61004794e-01,5.14840186e-01],[1.02319144e-01,-1.82673596e-02,-4.15522903e-01]],[[-3.14817220e-01,1.46576062e-01,-9.93438214e-02],[2.07951218e-01,2.39579417e-02,-5.51405370e-01],[-4.77323890e-01,-3.93464029e-01,-5.26978731e-01]],[[4.13252115e-01,3.66572733e-03,6.62777722e-02],[1.16210496e+00,1.07484445e-01,4.34151322e-01],[-4.75566892e-04,-4.60909188e-01,3.27625871e-01]],[[-4.18315440e-01,1.18689284e-01,-7.61564076e-01],[9.21341330e-02,-6.43610597e-01,-1.28016219e-01],[-4.10387814e-01,7.51054287e-02,8.76110122e-02]],[[-3.89226414e-02,6.23152733e-01,1.79726541e-01],[-4.98307616e-01,7.22380161e-01,1.03337824e-01],[1.33879647e-01,-8.76703143e-01,1.09422110e-01]],[[3.13323975e-01,-5.39583623e-01,-8.16771090e-02],[-1.22187771e-01,-8.26673985e-01,-5.95167875e-01],[-1.91153318e-01,3.52900386e-01,-1.22255945e+00]],[[6.59155622e-02,3.75347912e-01,-4.10056859e-01],[2.20528156e-01,3.97252887e-01,1.83224976e-02],[-2.84556866e-01,2.19356060e-01,-6.24270320e-01]],[[1.02740133e+00,2.24487949e-02,1.52225494e+00],[-1.59135759e-01,6.17805004e-01,7.72078335e-01],[6.58660710e-01,-3.96771282e-01,3.51241618e-01]],[[2.60863215e-01,-9.38034728e-02,2.17021137e-01],[5.53891957e-01,4.25371885e-01,-2.40703881e-01],[6.86841547e-01,1.91786557e-01,-9.89807025e-02]],[[8.71608317e-01,-1.01533122e-01,-5.75238109e-01],[-4.50522900e-02,2.73168921e-01,-1.32412910e-01],[3.48235309e-01,-6.57866746e-02,-1.43833354e-01]],[[-3.20994586e-01,-4.52475756e-01,8.32526803e-01],[-3.15235108e-01,6.39231920e-01,2.03223884e-01],[2.01537967e-01,-8.59971404e-01,2.58092493e-01]],[[6.58536553e-01,-1.18690145e+00,-3.96774411e-01],[3.09299707e-01,1.30327165e-01,-5.33063948e-01],[3.09310824e-01,5.32555699e-01,-2.57479429e-01]],[[1.52478740e-01,2.41761252e-01,-5.87785058e-02],[-3.81365031e-01,3.55617821e-01,4.87175971e-01],[-3.92945051e-01,-1.69249400e-01,1.61342800e-01]],[[3.06478947e-01,3.84453863e-01,-2.44423285e-01],[-1.70016348e-01,3.96662980e-01,4.57126588e-01],[1.51309505e-01,4.41189140e-01,-4.63369876e-01]],[[3.49606156e-01,1.97453592e-02,3.89404386e-01],[5.24679065e-01,-5.31324983e-01,6.11459240e-02],[7.29292154e-01,-1.75280273e-01,2.51692310e-02]],[[2.23428711e-01,2.41916925e-02,-1.96825966e-01],[4.98524457e-01,-6.56566262e-01,5.17490864e-01],[2.32612565e-01,1.01709142e-01,-3.00996214e-01]],[[-7.24294633e-02,-4.70669955e-01,-2.97871977e-01],[2.42138237e-01,-6.74669743e-01,9.72953081e-01],[-6.00507677e-01,2.84703553e-01,-1.88368767e-01]],[[1.06415205e-01,-5.97391903e-01,-2.42697075e-01],[1.10970750e-01,-1.33583367e-01,-5.54061413e-01],[1.87093690e-01,1.89698506e-02,-2.97312051e-01]],[[-1.70066953e-01,-2.73438424e-01,-8.25604558e-01],[2.47939140e-01,-7.83334315e-01,-4.28880811e-01],[4.71811146e-01,-2.49133423e-01,-1.41447589e-01]]],[[[6.08506143e-01,6.47860706e-01,8.62739325e-01],[2.93853879e-01,5.58432102e-01,-2.80105650e-01],[-2.61006385e-01,4.16892469e-02,-1.39172390e-01]],[[-1.90316271e-02,-6.29233345e-02,1.05053395e-01],[-6.24522269e-01,4.59263831e-01,-2.55361110e-01],[-3.24936584e-04,7.31692970e-01,-3.15452754e-01]],[[-5.74175358e-01,5.26883863e-02,1.57670999e+00],[-3.11204523e-01,1.05107605e-01,3.54409486e-01],[2.17362911e-01,-3.05884123e-01,-4.86853085e-02]],[[-7.61691928e-02,-8.81428123e-01,-2.02664314e-03],[6.59179091e-01,6.70832574e-01,-7.19237924e-01],[3.07462662e-01,-6.02066994e-01,-6.25004351e-01]],[[-3.88893068e-01,-2.00648993e-01,-6.73259795e-02],[1.22676939e-01,1.72310352e-01,2.88490932e-02],[-8.55129585e-02,-4.50736612e-01,-3.49759161e-01]],[[-4.33168441e-01,2.10005008e-02,-9.34803337e-02],[-1.39550710e+00,-1.50101744e-02,3.26110572e-02],[2.12871537e-01,3.00818328e-02,-6.99564934e-01]],[[-4.30727869e-01,-3.04514747e-02,3.68130356e-01],[-3.90462279e-01,3.09653074e-01,-4.00723457e-01],[2.69185066e-01,-1.99508071e-01,-3.38921309e-01]],[[-6.55569434e-02,2.74473518e-01,-3.17966163e-01],[3.92263476e-03,5.44364154e-01,5.46086431e-01],[-3.15428644e-01,-8.17215204e-01,4.46117818e-01]],[[1.64182112e-01,4.06115115e-01,-4.09586430e-01],[1.58152103e-01,2.52914459e-01,-5.93601525e-01],[2.26056874e-01,-1.76590636e-01,-1.15017727e-01]],[[-4.95754220e-02,-1.99101806e-01,1.49430111e-02],[-2.48939276e-01,1.88034505e-01,-2.06861094e-01],[-2.69363731e-01,2.37298280e-01,8.98870304e-02]],[[-1.22292407e-01,-7.31205046e-02,-9.79474485e-01],[6.63285032e-02,9.11763869e-04,-4.93145496e-01],[-7.86897838e-01,1.54863268e-01,-1.25299752e-01]],[[-3.94733846e-01,-5.49804449e-01,2.96771646e-01],[-2.58674204e-01,-5.88489950e-01,1.54513672e-01],[7.56168246e-01,-1.68680310e-01,3.28107655e-01]],[[-1.70398474e-01,7.92652547e-01,-1.72475487e-01],[-3.78059484e-02,-4.85433117e-02,2.33822286e-01],[4.31038849e-02,3.57637137e-01,-2.64293134e-01]],[[1.22111268e-01,-1.56124607e-01,-1.86155140e-02],[1.92053154e-01,5.72030544e-01,2.52993703e-02],[5.55553854e-01,6.01495564e-01,-3.93581629e-01]],[[4.30070847e-01,-8.41417909e-02,-2.11642772e-01],[7.37119615e-01,1.48223072e-01,-5.00421762e-01],[-8.89784992e-02,1.33597001e-01,-2.29867939e-02]],[[-4.71453816e-01,-6.66834041e-02,4.62533766e-03],[8.08100939e-01,3.01898718e-01,-2.75280684e-01],[-2.21162066e-02,-2.19274551e-01,-3.91321987e-01]],[[5.14268160e-01,3.03222686e-01,7.03808516e-02],[-8.40278745e-01,7.08319396e-02,-6.37226760e-01],[6.30057871e-01,-1.74422160e-01,-2.44237468e-01]],[[-8.49337578e-01,-4.23297882e-01,-9.70303938e-02],[-2.59541631e-01,3.69732141e-01,2.68863112e-01],[8.86403397e-02,-2.05770686e-01,1.28845358e-02]],[[-1.91199318e-01,-7.68429995e-01,3.04948658e-01],[1.13756493e-01,-6.50133491e-01,-2.97184497e-01],[-6.98900223e-01,-6.66908860e-01,9.13399607e-02]],[[-2.82473773e-01,2.34023288e-01,2.90787313e-02],[-2.84568556e-02,5.76214910e-01,-3.57229948e-01],[-3.89072269e-01,-1.41593114e-01,-7.41428792e-01]],[[7.72376239e-01,3.09411734e-01,1.08542359e+00],[-4.24899906e-01,8.46243024e-01,1.33869827e-01],[6.66580558e-01,-6.28035307e-01,2.61198014e-01]],[[-1.24951363e+00,4.51786160e-01,1.64124370e-01],[-6.39184564e-02,-3.57187063e-01,-8.97568837e-02],[-1.81394935e-01,-7.73251504e-02,1.08986065e-01]],[[-2.62273587e-02,1.74198955e-01,-5.30990362e-01],[3.41284573e-02,5.07662352e-03,-1.64624289e-01],[5.63203394e-01,6.87619686e-01,-9.41143408e-02]],[[-3.99603754e-01,-9.42157879e-02,2.87188858e-01],[3.51296097e-01,-1.34120330e-01,1.48046717e-01],[7.04231858e-01,-1.33441254e-01,2.42332608e-01]],[[-1.08349431e+00,-1.01798490e-01,-1.22200707e-02],[-7.92665541e-01,3.56895298e-01,-4.45754588e-01],[-7.71718007e-03,-1.30635370e-02,3.37330788e-01]],[[-5.97160578e-01,1.44301414e-01,3.57737660e-01],[-1.19994462e-01,-3.29581976e-01,2.15855435e-01],[-1.34588540e+00,-1.27143860e+00,-1.44393250e-01]],[[1.04738235e+00,-4.60556634e-02,-8.16500187e-01],[-1.65827144e-02,9.66366708e-01,2.34586075e-02],[1.57602094e-02,-3.91202420e-01,-2.78145790e-01]],[[2.43830472e-01,-6.02606952e-01,-4.37550068e-01],[-3.01955849e-01,-5.58200717e-01,3.74007933e-02],[2.11835075e-02,-8.80477950e-02,5.33238411e-01]],[[2.62588244e-02,6.40429020e-01,5.65388262e-01],[3.16557705e-01,1.84798017e-01,-3.83527838e-02],[-2.01440498e-01,-2.21426442e-01,-1.39386654e-01]],[[9.11059007e-02,-5.82126677e-01,2.81893909e-01],[-8.81061256e-02,-4.38515216e-01,8.09945941e-01],[5.67096472e-01,3.36504996e-01,5.52354336e-01]],[[1.34510589e+00,-2.20323667e-01,6.99596524e-01],[-3.30194421e-02,-5.30486964e-02,-2.63462096e-01],[-2.25587696e-01,-4.29173946e-01,7.15494603e-02]],[[-9.51917320e-02,1.59466267e-01,3.50932747e-01],[1.45580284e-02,-5.29336929e-01,-2.86153853e-02],[-3.80988479e-01,-8.44272077e-01,2.75087744e-01]],[[3.53237689e-01,4.36199576e-01,-2.42220923e-01],[4.92482563e-04,6.73243225e-01,4.81263727e-01],[2.68829048e-01,-4.08795834e-01,1.93051212e-02]],[[-1.75790519e-01,-2.39824399e-01,-1.34346879e+00],[-8.16823006e-01,-4.06523496e-01,2.98970044e-01],[1.41186967e-01,-1.35085583e+00,5.41758537e-01]],[[-1.46455303e-01,-3.58460210e-02,-7.82581627e-01],[-1.94350570e-01,-4.09079701e-01,4.29748863e-01],[-1.41485587e-01,-5.90042114e-01,9.32214022e-01]],[[8.62722695e-01,1.73051491e-01,9.03576553e-01],[3.29680651e-01,-1.76582232e-01,-2.68119544e-01],[-1.90118197e-02,4.43004221e-01,6.48415267e-01]],[[-2.74568945e-01,-2.70311028e-01,-5.71668386e-01],[1.73241775e-02,-6.50237918e-01,-1.46406963e-01],[3.92360210e-01,-3.92786801e-01,4.77083087e-01]],[[7.83667490e-02,1.86622307e-01,4.40749705e-01],[-7.45800376e-01,6.89594820e-02,8.44300836e-02],[-1.50281608e-01,8.83933380e-02,-3.91897202e-01]],[[1.49484336e-01,1.00905634e-01,-6.52953684e-01],[-1.63103148e-01,-6.00913942e-01,-9.17995572e-02],[9.75027010e-02,-4.23312008e-01,5.67023382e-02]],[[-6.53843045e-01,-5.81976175e-02,5.19362032e-01],[-1.06938136e+00,2.58566201e-01,-9.39231142e-02],[6.05594218e-01,1.73613250e-01,8.74267936e-01]]],[[[-1.64667815e-01,-6.09623849e-01,-2.30622754e-01],[-6.09666407e-01,1.73986450e-01,-5.74454188e-01],[6.18020654e-01,2.09542096e-01,-8.93976092e-01]],[[-1.76121518e-01,4.57387835e-01,7.92068124e-01],[-1.04755044e+00,-3.14838976e-01,9.51516032e-02],[9.11091268e-01,4.85300124e-01,-2.18880534e-01]],[[4.51399446e-01,-7.66375780e-01,4.73200142e-01],[1.30675256e-01,-5.07732809e-01,-2.84011573e-01],[9.16457027e-02,4.85969573e-01,7.16954648e-01]],[[-3.35564673e-01,-2.88502909e-02,3.77466738e-01],[8.40581536e-01,3.86510432e-01,-4.87702876e-01],[4.01397189e-03,3.06339264e-01,-8.09991434e-02]],[[1.92644075e-01,-9.13383886e-02,-2.34632805e-01],[-6.19804621e-01,5.78890136e-03,7.33972371e-01],[6.10278845e-01,3.47675413e-01,-3.09520215e-01]],[[4.76038545e-01,8.99512321e-02,-3.64635110e-01],[-4.34794188e-01,-9.38341260e-01,-5.21419287e-01],[-2.78337896e-01,7.33959615e-01,5.58496177e-01]],[[7.74198711e-01,-5.31252623e-01,-6.69913948e-01],[-6.16375543e-03,-5.55257499e-02,2.77146883e-02],[-7.42387772e-01,4.76547003e-01,-3.74204934e-01]],[[3.88098300e-01,-8.56869102e-01,-4.91365314e-01],[5.17265916e-01,-5.84249087e-02,6.35229051e-01],[-2.49059498e-02,-4.61850345e-01,8.12082469e-01]],[[-5.26953697e-01,-1.92939132e-01,-1.73398003e-01],[-8.15890968e-01,-3.88641566e-01,-2.06132948e-01],[3.45345974e-01,8.65100801e-01,-2.09405050e-01]],[[-1.30977720e-01,-3.23504746e-01,2.38837138e-01],[2.16863275e-01,3.34463030e-01,-1.38906687e-01],[-1.18850812e-01,-3.60440388e-02,-3.46727312e-01]],[[-2.70721912e-01,3.80595148e-01,-7.71321058e-01],[5.26239201e-02,-5.26006341e-01,-4.90126684e-02],[-8.83256495e-01,-2.45056450e-01,-5.54038823e-01]],[[-7.11783051e-01,-5.46250284e-01,4.52162027e-01],[-4.15183306e-02,-2.13892370e-01,2.41636515e-01],[-6.53852895e-02,1.26538843e-01,5.64231165e-02]],[[2.08269387e-01,1.02945828e+00,1.69726819e-01],[-7.56377995e-01,-9.99527931e-01,8.76400471e-02],[-2.07909390e-01,5.91828823e-01,9.07793492e-02]],[[-6.12005532e-01,1.78404212e-01,7.10727394e-01],[2.05838174e-01,-8.76562655e-01,-1.33432284e-01],[-2.25681946e-01,-4.99432832e-02,-2.64754385e-01]],[[-3.41789097e-01,4.33725744e-01,-4.35810953e-01],[7.12217167e-02,-2.56606162e-01,1.15293615e-01],[-1.65975034e-01,-8.41821253e-01,-1.92154646e-01]],[[2.56528825e-01,-6.35101080e-01,-5.53001881e-01],[-1.69237882e-01,1.44002602e-01,4.06932145e-01],[6.22873902e-01,3.56214225e-01,-7.64078423e-02]],[[-4.54177052e-01,-4.10135061e-01,3.78534436e-01],[-1.49334741e+00,7.02965736e-01,8.99290666e-02],[-4.09810282e-02,-3.27929527e-01,7.34289456e-03]],[[-3.16615440e-02,2.10396037e-01,-7.59393501e-04],[-1.97541729e-01,4.21643704e-01,1.60123006e-01],[-1.55566633e-02,-8.03820908e-01,-1.57889098e-01]],[[1.42574251e-01,-5.22737741e-01,2.51251012e-01],[4.39822078e-02,-7.91592181e-01,-1.03449516e-01],[2.32227221e-01,-9.04259920e-01,1.11686669e-01]],[[-9.56492648e-02,-5.97316980e-01,5.61549902e-01],[8.69797692e-02,-1.01759398e+00,-7.90433466e-01],[2.54213929e-01,-2.07580358e-01,-7.20035195e-01]],[[-2.93650001e-01,-5.63856587e-02,6.42451882e-01],[4.40920964e-02,-2.24378915e-03,-1.19528502e-01],[2.20468938e-02,2.39544004e-01,-1.02611378e-01]],[[-3.57443154e-01,1.98854402e-01,-5.09664059e-01],[4.77576882e-01,-2.31586009e-01,3.23737055e-01],[-3.52078557e-01,6.64353728e-01,-2.70135045e-01]],[[-1.28660649e-01,-1.03992932e-01,-2.53249526e-01],[5.23939803e-02,6.42123260e-03,3.11401933e-01],[2.57957205e-02,-4.42975193e-01,3.42228979e-01]],[[-5.77404261e-01,-5.06621599e-01,1.54921800e-01],[-6.22036338e-01,5.04626155e-01,2.86316842e-01],[6.42168224e-01,2.64708370e-01,-7.07077205e-01]],[[3.65578979e-01,-2.19547480e-01,5.33522248e-01],[-9.67838168e-02,-2.01687634e-01,-8.75959814e-01],[2.19549432e-01,-3.26020390e-01,1.38272271e-01]],[[-1.13110252e-01,-3.48528355e-01,2.55126595e-01],[2.42224634e-01,-5.39843619e-01,-1.17256552e-01],[4.45841789e-01,-1.45307258e-01,-8.72135103e-01]],[[2.44229212e-01,2.16703862e-01,6.59205485e-03],[1.01440489e+00,3.36020857e-01,-3.20262730e-01],[9.05733824e-01,-2.16167405e-01,9.31872725e-01]],[[-8.49007666e-01,2.04637036e-01,-6.38576806e-01],[-1.49629846e-01,-2.78077334e-01,-1.43583441e+00],[4.98241633e-01,5.61814249e-01,-5.15497506e-01]],[[-6.92726895e-02,-3.26424539e-01,-2.58350343e-01],[2.13367969e-01,4.89380993e-02,3.75517547e-01],[-4.50280398e-01,-1.26200750e-01,1.28372712e-02]],[[-7.21370757e-01,2.09676951e-01,-8.99260268e-02],[3.85870375e-02,3.35515797e-01,8.01062137e-02],[2.88925439e-01,-4.79607254e-01,-1.01160109e-01]],[[-4.46810842e-01,-2.47379556e-01,-7.35933483e-02],[7.39593029e-01,-6.33361340e-01,3.66695136e-01],[3.54170173e-01,1.71685308e-01,2.56399930e-01]],[[-8.99050236e-02,5.84954977e-01,2.79120058e-01],[7.04279363e-01,3.14844787e-01,5.33368528e-01],[6.20708823e-01,-9.18063223e-02,-2.28891596e-01]],[[9.77425557e-03,-3.50978374e-01,-4.02793646e-01],[2.92006209e-02,-7.77857959e-01,-6.49708509e-01],[-2.80869335e-01,-2.51423001e-01,5.69860399e-01]],[[8.77574682e-02,4.21371549e-01,1.13410026e-01],[4.18885976e-01,2.31824353e-01,-9.31253672e-01],[-1.15278482e-01,-3.40567529e-01,-7.89515227e-02]],[[3.44149828e-01,2.74838775e-01,6.00395024e-01],[3.48218940e-02,-1.19129680e-01,2.38227054e-01],[5.24235547e-01,-5.52944064e-01,-2.36569807e-01]],[[6.44941777e-02,-2.42232174e-01,-3.64129424e-01],[3.79144967e-01,1.13282487e-01,-6.36319280e-01],[-2.01415613e-01,-2.70858765e-01,8.19194257e-01]],[[4.06656861e-01,-3.08084756e-01,-2.75886268e-04],[-2.70107269e-01,-5.00902832e-01,-8.31920356e-02],[-1.36845064e+00,1.17655165e-01,-8.72704685e-02]],[[2.12122664e-01,5.79037488e-01,-4.77332234e-01],[7.35692024e-01,-1.62738413e-01,3.64024401e-01],[-4.61231202e-01,-4.69970584e-01,8.43240991e-02]],[[-4.90985513e-02,-2.76395202e-01,-1.28977165e-01],[2.04135060e-01,4.22708541e-01,2.84296602e-01],[-5.49202450e-02,6.32451653e-01,-7.70684600e-01]],[[-4.31410700e-01,-4.40122455e-01,1.45398647e-01],[-3.22965801e-01,4.37177360e-01,1.14315838e-01],[1.32054403e-01,4.24254894e-01,-4.45248604e-01]]],[[[8.47143829e-01,3.41610573e-02,2.59965569e-01],[1.93498433e-01,6.38514042e-01,2.09369466e-01],[4.30761069e-01,1.75621584e-01,-5.07026196e-01]],[[-2.50640601e-01,-8.11621785e-01,-4.63112205e-01],[-6.25797033e-01,1.88646972e-01,1.79318767e-02],[3.28145474e-01,7.22168386e-02,-6.67536333e-02]],[[-2.50795007e-01,4.19270515e-01,3.83732557e-01],[-2.10609585e-01,3.09171110e-01,1.43934503e-01],[-1.59715079e-02,1.55295596e-01,-9.37284976e-02]],[[-5.45085907e-01,-2.19057739e-01,-7.90930539e-02],[1.55938998e-01,-8.22720170e-01,-4.08117205e-01],[-7.23606050e-01,2.10444015e-02,-6.37609661e-02]],[[-2.07107104e-02,1.82133019e-01,-4.70418483e-01],[-5.35326339e-02,-1.93422079e-01,-2.06163511e-01],[-3.94198537e-01,-5.48419133e-02,-1.64973110e-01]],[[-3.06483954e-01,7.76176453e-01,6.79220319e-01],[-1.97201774e-01,-7.94272050e-02,-1.88927695e-01],[-3.61014456e-01,-6.66966319e-01,3.33238870e-01]],[[-2.44048208e-01,5.82489550e-01,-1.08873904e+00],[5.39369062e-02,3.21456999e-01,2.78186560e-01],[-3.38996679e-01,3.55434686e-01,8.21987167e-02]],[[-1.11657751e+00,2.47917324e-01,1.96121093e-02],[4.98410650e-02,2.26108864e-01,5.97240217e-02],[3.58601809e-01,1.66193515e-01,2.69972920e-01]],[[2.72664636e-01,9.63767618e-02,-1.27615090e-02],[6.69813514e-01,-1.10731892e-01,-1.12156108e-01],[-2.37708598e-01,-5.66405177e-01,1.99555784e-01]],[[-9.10321951e-01,-4.99046952e-01,3.59477028e-02],[4.88499135e-01,3.88409011e-02,7.52566278e-01],[1.87291518e-01,-3.49492952e-02,-2.49941438e-01]],[[-4.81073201e-01,-3.24033767e-01,2.28960082e-01],[-5.35909738e-03,5.12759507e-01,-2.37263769e-01],[7.09604800e-01,-4.66193780e-02,9.38181430e-02]],[[-5.63743949e-01,1.02538621e+00,-5.36666811e-01],[1.08987108e-01,3.00690740e-01,-2.39981964e-01],[2.71709949e-01,2.16516852e-01,-5.31895697e-01]],[[1.03379774e+00,-2.82058477e-01,3.12823027e-01],[1.63841963e-01,-1.33189157e-01,7.16393650e-01],[-7.04435483e-02,-1.37326181e-01,-6.69971466e-01]],[[-7.26108730e-01,2.21710965e-01,-1.81330919e-01],[2.66590059e-01,1.96328446e-01,3.70788753e-01],[-1.00020826e+00,3.58012468e-01,-3.24301660e-01]],[[4.92700398e-01,-2.30190486e-01,-4.61082906e-01],[-3.50469470e-01,1.66318547e-02,-2.16937542e-01],[-6.80064261e-01,-1.29480517e+00,-3.72375071e-01]],[[2.95893401e-01,3.40234339e-02,-3.68544638e-01],[-1.52477756e-01,-1.17242500e-01,5.96424878e-01],[7.11749077e-01,-6.35379195e-01,-1.84457824e-01]],[[-5.30826867e-01,2.33287096e-01,2.88749337e-01],[9.52265859e-02,2.06927389e-01,9.17058960e-02],[9.05267000e-01,-3.77593637e-01,4.62512150e-02]],[[1.30285978e-01,2.73483813e-01,9.50174406e-02],[4.35388774e-01,5.11894166e-01,-4.04425055e-01],[-5.55052578e-01,-3.59651595e-02,3.56415719e-01]],[[4.41887937e-02,-7.57180303e-02,3.59574497e-01],[-6.68635130e-01,6.44450188e-01,-2.55723268e-01],[-4.81069423e-02,-2.82446533e-01,4.64922279e-01]],[[5.35159409e-01,6.66987300e-02,1.75821006e-01],[4.91244465e-01,2.93395758e-01,-7.97364652e-01],[5.43203056e-01,-1.17706048e+00,5.90187684e-02]],[[-6.19824290e-01,-1.79543570e-01,1.35463737e-02],[-1.50558293e-01,7.34458685e-01,-7.33839631e-01],[-9.55560058e-02,-6.16029084e-01,1.36191905e-01]],[[1.68068349e+00,4.65197682e-01,1.08247235e-01],[8.31144214e-01,5.34187794e-01,3.37027103e-01],[1.01168466e+00,-5.55850208e-01,1.08390510e-01]],[[7.37329364e-01,-9.88395870e-01,1.82741520e-03],[5.82225978e-01,2.90800452e-01,2.25713521e-01],[1.33942008e-01,-4.05322045e-01,3.17970723e-01]],[[2.71500528e-01,-2.74835110e-01,-3.14376622e-01],[-1.01837151e-01,6.12292945e-01,1.97842509e-01],[-1.09669316e+00,-4.28431064e-01,-5.34505546e-01]],[[7.23328412e-01,-4.42286938e-01,-2.06914451e-02],[-5.49331188e-01,5.09904861e-01,-2.78143823e-01],[5.94100714e-01,-2.72875071e-01,-3.35400045e-01]],[[8.14859927e-01,3.93286645e-01,-6.13896966e-01],[2.21693531e-01,-6.61251247e-01,2.74804592e-01],[-2.32716158e-01,9.55726266e-01,1.40282184e-01]],[[2.41262630e-01,2.19221041e-01,-6.16168864e-02],[2.07949460e-01,-3.72874916e-01,2.34386578e-01],[-5.84476829e-01,-7.98186183e-01,-5.18026650e-01]],[[2.13623300e-01,6.00572862e-02,-3.06997955e-01],[4.11897302e-01,2.48765022e-01,1.86191335e-01],[6.76003873e-01,1.44190555e-02,-3.83914143e-01]],[[5.76379776e-01,-2.56342828e-01,7.40423650e-02],[2.11773023e-01,-6.40018165e-01,3.71740937e-01],[3.57207991e-02,-1.69230208e-01,4.86266911e-01]],[[-6.72828913e-01,-9.25696343e-02,7.38867791e-04],[3.75346869e-01,7.59923697e-01,2.28159856e-02],[-5.83667874e-01,-9.66576487e-02,-8.99193227e-01]],[[-9.03834403e-01,-5.05204499e-02,1.10395692e-01],[2.95277592e-02,2.37224594e-01,-3.88219416e-01],[-5.41567318e-02,3.43505852e-02,-2.29916304e-01]],[[9.18739200e-01,-3.63108099e-01,1.07742384e-01],[4.68341678e-01,-6.29198313e-01,-1.74477207e-03],[4.95410413e-01,7.92810917e-02,8.34823430e-01]],[[-1.25107095e-01,-1.63431942e-01,-2.25672200e-01],[-4.31602210e-01,-6.89613461e-01,-5.24552107e-01],[1.56317711e-01,-4.81212407e-01,4.71987158e-01]],[[1.66665673e-01,-4.32673216e-01,-2.24794388e-01],[-2.01189324e-01,2.31271356e-01,-4.91664618e-01],[-3.44124138e-01,1.82831660e-01,-7.35971183e-02]],[[-2.95401424e-01,-4.16356444e-01,-1.70091018e-02],[2.69448727e-01,-3.31362754e-01,-7.82591760e-01],[8.25218916e-01,-2.78129548e-01,-1.57462936e-02]],[[4.44833457e-01,-6.10553741e-01,2.10521370e-02],[-5.20447373e-01,-6.91348374e-01,4.12588298e-01],[-1.27524912e+00,-4.96856630e-01,3.71052861e-01]],[[-2.08281502e-01,-2.41241250e-02,-6.44539416e-01],[-3.02102894e-01,2.01494262e-01,2.82770187e-01],[5.99506423e-02,-8.09828043e-01,-2.71450821e-02]],[[-2.55689055e-01,-1.49780333e-01,1.26390338e+00],[-1.54340938e-01,2.21518725e-02,2.96757728e-01],[6.16277575e-01,-6.92977726e-01,-6.36743903e-01]],[[-9.69752297e-02,-2.78800160e-01,2.60922909e-01],[1.02669477e+00,-3.51307482e-01,6.13285959e-01],[4.57459271e-01,2.79742599e-01,1.23237483e-02]],[[2.18999624e-01,-8.34137261e-01,2.47310683e-01],[4.53675389e-01,-1.54029548e-01,6.79033548e-02],[-1.74307004e-01,5.24082065e-01,1.07512303e-01]]],[[[9.35731769e-01,5.14312029e-01,-1.83123410e-01],[2.46850066e-02,-6.45150661e-01,4.90855932e-01],[1.24074385e-01,-1.85093924e-01,-1.58434007e-02]],[[-2.43178993e-01,2.72589743e-01,1.88312039e-01],[-1.66488364e-01,5.96069157e-01,-8.40852708e-02],[-3.18917394e-01,5.44358015e-01,6.80626214e-01]],[[2.62613297e-01,-1.96217358e-01,5.84276557e-01],[-7.94723094e-01,-4.60729152e-01,4.89020914e-01],[5.41004464e-02,-2.49746427e-01,1.79186568e-01]],[[6.28885627e-02,4.22179908e-01,1.25862226e-01],[3.75920013e-02,-8.51582587e-01,-4.44749713e-01],[-1.15465355e+00,1.17502317e-01,2.46088877e-01]],[[-1.31826848e-01,-7.27631688e-01,6.47274554e-01],[6.06825352e-02,3.24274212e-01,1.99821722e-02],[-8.78735036e-02,-5.40617466e-01,-8.68536532e-01]],[[4.37479585e-01,-1.34413287e-01,-2.97508091e-01],[1.21622570e-02,-4.98322129e-01,6.46140754e-01],[5.55824399e-01,9.59041238e-01,-2.56595369e-02]],[[1.41354222e-02,1.28454402e-01,-6.02540531e-05],[-2.90326655e-01,6.62839890e-01,-1.09158874e-01],[1.51724920e-01,-4.04259145e-01,8.42294246e-02]],[[-1.19290166e-01,-1.56341255e-01,5.80991447e-01],[3.57871205e-01,-5.81704676e-02,1.27921686e-01],[-1.52867958e-01,-4.90836620e-01,3.87320399e-01]],[[-7.82663226e-01,1.43051967e-01,-4.49816525e-01],[5.72182983e-02,-7.83619285e-02,3.80386025e-01],[2.31540293e-01,2.03007877e-01,-2.30012432e-01]],[[-1.31943807e-01,-2.00376794e-01,3.12354296e-01],[-2.37897709e-01,-3.31667185e-01,-1.58668578e-01],[-2.97016233e-01,-4.55354005e-01,-4.03544277e-01]],[[-2.10903123e-01,3.11326772e-01,-1.10495046e-01],[-3.13967317e-01,2.00874269e-01,-1.94410324e-01],[-1.92247689e-01,-6.68323994e-01,6.14002585e-01]],[[-1.56606197e-01,-1.53148592e-01,-3.67792249e-01],[4.47935641e-01,-2.02006966e-01,-1.91558480e-01],[-2.55265296e-01,4.88506295e-02,-6.07247472e-01]],[[-4.71638411e-01,-4.03647065e-01,4.26964313e-01],[-6.49945363e-02,-9.81438383e-02,-6.28366947e-01],[-1.33121479e+00,6.47365570e-01,3.60813469e-01]],[[-8.23580682e-01,6.31334960e-01,6.94583118e-01],[-1.73247047e-03,4.88228530e-01,-7.86778510e-01],[2.38340691e-01,9.61177528e-01,-1.45939916e-01]],[[6.43305555e-02,-6.74158216e-01,-6.47017300e-01],[-1.15243360e-01,5.03297806e-01,1.42922297e-01],[-5.67196846e-01,1.20239757e-01,-2.92214990e-01]],[[-3.82761747e-01,4.61144447e-01,7.78020740e-01],[-4.50553149e-01,-7.72668242e-01,2.05795228e-01],[-7.84050524e-01,-8.14238906e-01,3.17341268e-01]],[[-2.72406667e-01,-5.92442900e-02,2.75154203e-01],[2.87196636e-02,-4.44431573e-01,1.86959818e-01],[-2.47090831e-01,1.22257277e-01,-1.37237012e-01]],[[-5.44697583e-01,2.92645842e-01,6.81427261e-03],[4.92964149e-01,4.71950732e-02,-4.01879072e-01],[2.32847631e-01,5.90103924e-01,2.16823608e-01]],[[2.89022386e-01,7.72436261e-01,4.66968268e-01],[-3.54768485e-01,4.25907582e-01,-1.93182230e-01],[2.11837620e-01,-1.45833613e-03,-1.08855776e-01]],[[-2.51659900e-01,-8.81503344e-01,-2.35457942e-01],[3.23287696e-01,4.64487255e-01,7.49800578e-02],[-6.19015694e-02,-4.15042013e-01,-7.02042460e-01]],[[-4.09074485e-01,4.89313126e-01,1.63411462e+00],[-2.73293614e-01,6.99766874e-01,-1.99088007e-01],[1.12671651e-01,3.70764822e-01,4.34874505e-01]],[[9.18703139e-01,5.17901242e-01,9.09633711e-02],[5.90091765e-01,-2.97860175e-01,-5.38040102e-01],[-8.10510278e-01,-8.33686054e-01,-5.80940783e-01]],[[-2.38032401e-01,5.66573739e-01,-4.90338027e-01],[-1.67507678e-01,-1.14478946e-01,-8.33791494e-02],[4.57617581e-01,7.67256767e-02,2.31622290e-02]],[[-5.04402936e-01,2.58251458e-01,1.86149925e-02],[-2.97619253e-01,-2.42377110e-02,-6.14955366e-01],[-1.00483716e+00,3.27231467e-01,-6.30302787e-01]],[[1.22918352e-01,-4.42911744e-01,3.53767499e-02],[6.77353859e-01,-2.43722810e-03,3.64318863e-02],[3.66186798e-01,-1.27818063e-01,-6.66499674e-01]],[[2.61074811e-01,-9.23087239e-01,6.28789425e-01],[3.11174244e-01,-2.25962251e-02,-1.15395926e-01],[-9.66356039e-01,-4.55983728e-01,2.57239699e-01]],[[-3.31991076e-01,4.88493413e-01,-6.45763636e-01],[-1.82472378e-01,4.14037734e-01,-3.36924314e-01],[1.62119091e-01,3.94535422e-01,-1.75157905e-01]],[[-6.28794849e-01,6.23764455e-01,-1.31248832e-01],[-2.56177653e-02,3.20332617e-01,-1.88338831e-01],[2.01452568e-01,8.36894691e-01,-2.87396699e-01]],[[2.60255337e-01,-6.69826418e-02,3.46474171e-01],[2.33772844e-01,-1.01430953e+00,1.56793058e-01],[-6.92817032e-01,-6.22334898e-01,3.57439876e-01]],[[-3.05223495e-01,4.67696548e-01,-5.45424409e-02],[2.11844474e-01,-4.39124405e-01,8.84180330e-03],[-8.18248987e-01,2.52541006e-01,2.95803752e-02]],[[-2.89208949e-01,3.83418798e-01,3.57659668e-01],[-1.23047721e+00,1.81994602e-01,1.04568578e-01],[-6.05203629e-01,2.23312214e-01,-6.60858870e-01]],[[-1.32853374e-01,-3.94793510e-01,2.49328360e-01],[-3.00200999e-01,1.48087934e-01,2.18230067e-03],[-4.05081451e-01,2.02108875e-01,-4.28828970e-02]],[[7.89349675e-01,8.49805355e-01,-3.59742999e-01],[3.39671150e-02,1.80223763e-01,5.13148010e-02],[-3.83428246e-01,-1.07223228e-01,4.22478288e-01]],[[2.22950280e-01,5.41326761e-01,-4.61916506e-01],[2.88047373e-01,-6.79176450e-01,6.55374169e-01],[6.61953092e-01,-9.59113359e-01,1.04434490e+00]],[[-2.06350163e-01,-2.66069949e-01,-6.44980133e-01],[1.17198952e-01,-4.85333651e-01,1.15770996e-01],[2.71785080e-01,-2.91625053e-01,1.00841737e+00]],[[-1.82546198e-01,-7.22866040e-03,-5.72180092e-01],[-3.78439039e-01,4.85950232e-01,3.48716229e-01],[7.92780891e-02,-1.76716037e-02,2.43945941e-02]],[[2.20517755e-01,4.31874096e-02,-3.05375963e-01],[-2.36974478e-01,-1.84682593e-01,-5.24673283e-01],[-2.53200293e-01,-3.84205393e-02,4.22663122e-01]],[[3.26727748e-01,1.01801611e-01,6.47792459e-01],[2.40221858e-01,7.09002972e-01,1.52086809e-01],[2.74291337e-01,-2.68359065e-01,-3.05862725e-01]],[[-2.94677138e-01,3.99672717e-01,4.58354264e-01],[3.23821098e-01,8.53338540e-02,-8.36103261e-01],[-2.86773860e-01,-1.77503571e-01,-7.84510195e-01]],[[-1.23250239e-01,4.86290425e-01,6.51449680e-01],[-3.93335909e-01,-4.99284565e-01,-9.10606459e-02],[-7.47161806e-02,-5.28598428e-01,-6.11788869e-01]]],[[[-6.64838031e-02,3.27778876e-01,-5.90653419e-01],[2.70036189e-03,-8.38244498e-01,2.22002149e-01],[-2.14618847e-01,-4.48182702e-01,7.72209823e-01]],[[-1.54857650e-01,8.57758075e-02,8.26308250e-01],[2.09985420e-01,-7.08194017e-01,6.28294826e-01],[5.22522867e-01,-5.28274357e-01,-1.32855713e+00]],[[8.11320394e-02,1.04582715e+00,6.33704185e-01],[-4.23622727e-01,-7.48130798e-01,-5.35541832e-01],[-1.58404484e-02,-7.38173053e-02,4.15130377e-01]],[[4.54446435e-01,8.45272169e-02,-1.08403063e+00],[1.27438700e+00,-1.88961476e-01,3.62415493e-01],[7.46810317e-01,-9.30550635e-01,-1.77607670e-01]],[[3.89238656e-01,4.92943078e-01,-8.50181282e-01],[7.90411755e-02,-1.80385292e-01,3.85530949e-01],[-1.83040708e-01,5.23119830e-02,5.38898706e-01]],[[2.34897286e-01,3.93349707e-01,-1.72915533e-01],[-3.63312304e-01,-1.31572470e-01,1.41079217e-01],[3.57004665e-02,4.76805717e-02,-2.88428247e-01]],[[1.67407878e-02,-4.51359630e-01,-7.51002669e-01],[9.57107604e-01,-2.04775691e-01,1.66385934e-01],[-1.29199535e-01,3.09816837e-01,-1.43691108e-01]],[[3.85106474e-01,5.68300128e-01,-4.18701321e-01],[-2.67520010e-01,-7.25635216e-02,3.98406178e-01],[2.71864533e-01,7.50932768e-02,5.09330571e-01]],[[-1.16032116e-01,1.86000094e-01,5.31851470e-01],[-1.02154160e+00,6.58148170e-01,-1.67829812e-01],[-2.06531376e-01,2.32754394e-01,4.50029761e-01]],[[-7.32621372e-01,3.64282504e-02,-9.64523405e-02],[-1.35542393e-01,3.63013521e-02,2.12730616e-01],[-1.03081381e+00,3.72547433e-02,-2.96647102e-01]],[[-9.97033417e-02,-1.15747966e-01,-8.46896693e-03],[3.35947305e-01,2.28907928e-01,-6.10487700e-01],[2.68044826e-02,-9.59739685e-02,3.01841348e-01]],[[8.04555058e-01,7.65970528e-01,1.41465932e-01],[-4.82317537e-01,1.28936782e-01,4.25196290e-01],[-3.05721134e-01,4.37294662e-01,1.51022300e-01]],[[-2.57574916e-01,-6.32294044e-02,2.81616092e-01],[-7.34194219e-01,3.35360020e-01,5.36805391e-02],[1.18043460e-01,6.33642524e-02,1.22789454e+00]],[[-2.60949954e-02,7.01850206e-02,5.02584577e-01],[-9.58359689e-02,3.30837399e-01,-5.53001091e-02],[-2.85412580e-01,2.96055108e-01,-9.78701189e-02]],[[-2.90043503e-01,5.99492118e-02,-2.65149083e-02],[-8.70093763e-01,-2.09990039e-01,1.99994966e-01],[1.23868272e-01,-5.82198918e-01,-2.31324568e-01]],[[-1.12329948e+00,-6.17767096e-01,-5.99448562e-01],[-2.69408226e-01,-2.21125767e-01,4.53764319e-01],[-4.38199431e-01,-6.70334399e-01,-4.04201627e-01]],[[5.41750312e-01,3.34168166e-01,-1.26231030e-01],[2.97856748e-01,-9.17208474e-03,3.38004529e-01],[-6.34787917e-01,8.27442557e-02,-5.53114355e-01]],[[7.92115986e-01,-2.28538096e-01,-1.54542848e-01],[-1.56441256e-01,2.37146527e-01,-7.09754169e-01],[-3.23308021e-01,-7.85032570e-01,1.89960033e-01]],[[6.61303461e-01,-1.51688069e-01,2.41690964e-01],[6.80080131e-02,5.49446821e-01,8.82283449e-01],[-2.42926851e-01,-4.21243720e-02,-6.14622235e-02]],[[2.36571860e-02,-2.90002853e-01,-2.41338655e-01],[-6.13484681e-01,8.61925259e-02,-4.35033143e-01],[-2.08828971e-02,8.37221220e-02,3.70353550e-01]],[[-2.04700053e-01,-4.37526196e-01,1.34439349e+00],[3.97385120e-01,-6.88072667e-02,3.79419923e-01],[-3.26080292e-01,5.14351606e-01,2.90865570e-01]],[[3.20915282e-01,2.27006629e-01,-2.11619973e-01],[-2.72387743e-01,-2.02152610e-01,-3.27026725e-01],[-5.45868874e-01,3.91679764e-01,-2.59319842e-01]],[[-3.98369879e-01,2.98460364e-01,-4.21186984e-01],[1.28190309e-01,-4.16159444e-02,-1.94522411e-01],[-1.55789286e-01,4.21348512e-01,1.86123520e-01]],[[-7.99952596e-02,-2.36838907e-01,7.26092994e-01],[2.77853668e-01,3.11320961e-01,-2.49047399e-01],[5.07600605e-01,-1.51472852e-01,1.19506556e-03]],[[5.55770844e-02,-4.75141793e-01,4.54757273e-01],[2.51863413e-02,1.45296395e-01,2.20553428e-01],[-5.65879464e-01,1.49937063e-01,-1.78189993e-01]],[[4.32214402e-02,-3.93789262e-01,5.31629980e-01],[1.01370715e-01,-7.44623482e-01,6.91637158e-01],[-1.44554004e-02,3.80355924e-01,5.91277421e-01]],[[-2.62635946e-02,-7.52277195e-01,-2.19458029e-01],[5.18958628e-01,-3.12979460e-01,1.00358546e+00],[-7.67582953e-02,-9.95267704e-02,-7.23571718e-01]],[[-6.03565276e-01,-2.91905224e-01,2.61955351e-01],[-3.64861906e-01,5.38039878e-02,1.15609668e-01],[-2.03257114e-01,7.78662026e-01,-8.73192012e-01]],[[9.49289352e-02,4.19519007e-01,1.05393374e+00],[2.11405560e-01,-2.07407847e-01,-4.23790604e-01],[1.95283309e-01,5.27387261e-01,-2.57704049e-01]],[[-1.09323680e-01,3.34578633e-01,6.39901221e-01],[-4.15536910e-01,-2.19822615e-01,1.22784004e-01],[-6.04651213e-01,-2.68754512e-01,9.55758274e-01]],[[2.05829203e-01,-5.81797779e-01,4.67382520e-01],[8.70378971e-01,-9.93467808e-01,-3.04129958e-01],[2.04556599e-01,-7.13033974e-02,3.27811897e-01]],[[7.17299938e-01,-7.38554776e-01,-4.96532619e-01],[-1.47012234e-01,1.78913444e-01,9.15435076e-01],[2.46403351e-01,-1.18460037e-01,7.07383513e-01]],[[-2.23050952e-01,2.14840367e-01,9.50437188e-02],[5.01920164e-01,3.83599699e-01,-5.69877885e-02],[-9.53571498e-02,3.51339787e-01,-1.14142513e+00]],[[-1.23750694e-01,-4.98979688e-01,1.33514270e-01],[6.31262481e-01,-7.70881414e-01,3.20252776e-01],[6.37499392e-01,-2.56644785e-01,4.32893336e-01]],[[-4.30616140e-01,-4.75961804e-01,-8.29082370e-01],[1.18734814e-01,-1.58839405e-01,2.15619713e-01],[-2.08108038e-01,4.21581864e-01,7.10539639e-01]],[[-1.76847339e-01,-3.65984797e-01,9.92964804e-01],[2.35301599e-01,-5.11600435e-01,6.54406965e-01],[9.40718949e-01,-4.21068668e-01,-5.13379753e-01]],[[-3.15453974e-03,-5.88355422e-01,5.10963313e-02],[5.11146307e-01,7.61832148e-02,9.22530651e-01],[5.42124033e-01,-4.89987582e-01,4.51141357e-01]],[[-4.68364477e-01,-6.73778504e-02,-1.23289548e-01],[-3.23882997e-02,-3.81191850e-01,4.96692389e-01],[3.75290990e-01,6.13048337e-02,1.12355936e+00]],[[-1.26331657e-01,-5.12786508e-01,6.24633551e-01],[7.42805362e-01,1.23621929e+00,1.89411536e-01],[4.40148592e-01,-7.93045044e-01,-2.46631846e-01]],[[2.87482440e-01,-2.51121938e-01,7.66425133e-02],[7.60188282e-01,1.48467839e-01,-1.79985523e-01],[4.05103564e-02,-2.14523003e-01,3.26107889e-01]]],[[[1.56043172e+00,1.17321894e-01,4.93134707e-02],[8.99806082e-01,-6.66484535e-01,3.20579976e-01],[-3.26237530e-01,1.97242156e-01,-2.29995921e-02]],[[3.38626087e-01,1.44244209e-01,4.09718573e-01],[-2.12958068e-01,2.87809849e-01,1.84334323e-01],[-3.01164567e-01,2.02125385e-01,9.91417244e-02]],[[1.49454027e-01,-5.04160762e-01,4.31123108e-01],[4.64938469e-02,-1.51379913e-01,6.55776262e-01],[3.53183866e-01,2.57758498e-01,-7.74660334e-02]],[[1.63179368e-01,-2.89556265e-01,-1.21041797e-01],[4.47086424e-01,2.21004561e-01,-1.38487443e-01],[8.93033966e-02,-2.02275336e-01,-5.52885056e-01]],[[6.80266097e-02,-3.31551313e-01,1.18411452e-01],[-5.91513097e-01,-7.10440129e-02,8.74943852e-01],[-2.98307091e-01,2.53374517e-01,5.90940118e-01]],[[-3.22360694e-02,2.79636502e-01,-4.23150450e-01],[6.77448660e-02,3.82151872e-01,3.24510217e-01],[-3.44564706e-01,9.32191432e-01,9.45217311e-01]],[[1.20508097e-01,1.50775746e-01,-3.41855623e-02],[1.37047261e-01,7.25728273e-01,1.30764589e-01],[1.51046500e-01,1.46210775e-01,4.22910661e-01]],[[-4.65052456e-01,-2.20447034e-01,6.52963817e-01],[-7.40514398e-02,5.53837121e-01,2.41924107e-01],[-3.34403031e-02,6.43405139e-01,1.74485177e-01]],[[-3.11706632e-01,6.72060907e-01,1.03983633e-01],[-4.42234844e-01,4.64418948e-01,-1.14708440e-02],[-5.15845001e-01,1.08081508e+00,3.50347668e-01]],[[1.87864676e-01,2.67066807e-01,-4.52670336e-01],[3.66391957e-01,-2.79018134e-01,1.37667775e-01],[-1.70871413e+00,8.93021896e-02,-2.23524645e-01]],[[-7.68580064e-02,1.73210233e-01,1.51938945e-01],[5.68958402e-01,-4.30276133e-02,-2.70626575e-01],[1.50014520e-01,2.23259985e-01,1.24842271e-01]],[[3.59228015e-01,-3.89491081e-01,8.37491393e-01],[3.30206782e-01,2.84932554e-01,-2.46592283e-01],[1.95380241e-01,-8.50908816e-01,2.57141948e-01]],[[6.53599024e-01,2.72762507e-01,-2.08791047e-01],[4.12939698e-01,-3.19623262e-01,1.89002119e-02],[2.17718706e-01,9.40687116e-03,-3.54511775e-02]],[[3.61348987e-01,3.09725970e-01,3.45926993e-02],[6.46317899e-01,-4.45081115e-01,-2.61925131e-01],[1.61745369e+00,-3.93896490e-01,-3.01111758e-01]],[[8.43019664e-01,1.41249582e-01,8.57630551e-01],[7.34338939e-01,6.93178549e-03,-1.52503088e-01],[1.09533620e+00,1.80055425e-01,-6.14522919e-02]],[[-3.49108130e-01,3.72249514e-01,4.18592185e-01],[1.00289673e-01,2.08967760e-01,5.74047565e-01],[1.73802733e-01,5.86092293e-01,3.74406606e-01]],[[-6.88126832e-02,-3.53889391e-02,1.49350747e-01],[-3.06519330e-01,2.11382955e-01,1.57986581e-01],[-1.67842972e+00,4.60922599e-01,-1.07801720e-01]],[[-3.86665076e-01,-1.34355144e-03,2.83770561e-01],[-2.24060103e-01,-1.76871538e-01,1.09421508e-02],[2.01557547e-01,-2.85022140e-01,3.41178775e-01]],[[1.57908037e-01,7.08080977e-02,5.33055775e-02],[1.28475025e-01,1.58614546e-01,3.60104144e-01],[-2.02449486e-01,7.68841624e-01,-2.61185288e-01]],[[-9.19585675e-03,-1.01986349e+00,6.14271522e-01],[-1.79340020e-01,-1.28949553e-01,-3.15497577e-01],[-2.44390056e-01,-1.37473792e-01,-2.33801827e-01]],[[9.52210426e-01,3.02089423e-01,1.33468419e-01],[4.32403642e-04,-2.61608958e-01,3.84934366e-01],[3.97852957e-01,3.35170805e-01,6.87528968e-01]],[[-1.21232891e+00,1.49770588e-01,1.89072028e-01],[-7.64417052e-01,5.69175720e-01,-4.57619391e-02],[4.03171599e-01,2.65731096e-01,-7.89949968e-02]],[[-1.11249559e-01,-6.32636309e-01,-5.76666176e-01],[3.92928809e-01,1.05192102e-01,-5.94564021e-01],[3.73556346e-01,-6.22395217e-01,-8.32568884e-01]],[[2.07507327e-01,4.07686710e-01,-5.53424619e-02],[4.81690079e-01,-6.51230365e-02,1.94994539e-01],[1.75600052e-02,-1.47728205e-01,-5.17318010e-01]],[[-4.72788841e-01,-2.60061445e-03,-1.95726901e-01],[-2.42895022e-01,-6.00241497e-02,1.20102465e-01],[-4.63449657e-01,-3.72702926e-02,-1.75331816e-01]],[[-4.93786395e-01,-6.39833033e-01,-1.20977983e-01],[4.09796059e-01,-5.22669911e-01,-5.77998936e-01],[4.43305761e-01,8.49389732e-01,8.45089018e-01]],[[4.24889117e-01,-4.22647074e-02,3.91000807e-01],[5.57379365e-01,-7.64615357e-01,-3.17427397e-01],[7.87251711e-01,2.22993895e-01,7.18310952e-01]],[[-5.33793926e-01,-1.25864148e-01,4.00239885e-01],[8.45764577e-02,-3.90085012e-01,-5.20757749e-04],[6.62247092e-02,3.74150932e-01,-4.31764245e-01]],[[-6.07777894e-01,1.08167797e-01,9.74538684e-01],[2.74475485e-01,-4.52610478e-02,5.46470523e-01],[-5.68701982e-01,7.82770336e-01,6.81409687e-02]],[[1.06686480e-01,-3.42282921e-01,-1.98508590e-01],[4.30250704e-01,-1.59978017e-01,1.73134431e-01],[8.99847686e-01,-2.88827628e-01,2.74016947e-01]],[[1.67260861e+00,3.81884307e-01,-7.86519982e-03],[-2.85651851e-02,-3.99124563e-01,3.31627190e-01],[1.05812289e-01,3.15769523e-01,1.54750645e-01]],[[4.36710060e-01,-7.64187751e-03,-1.04621969e-01],[-3.10044847e-02,-9.15141180e-02,-4.29641813e-01],[-1.20630473e-01,-1.16146304e-01,-1.50073513e-01]],[[3.37379128e-01,-4.70817864e-01,-1.96970627e-01],[3.25092405e-01,1.19622648e-01,-2.54787326e-01],[4.77792583e-02,6.54005289e-01,-3.65714490e-01]],[[-5.26344717e-01,-9.56018567e-01,2.22812325e-01],[-5.46086788e-01,-5.58328927e-01,-4.56445634e-01],[6.07766032e-01,7.38508478e-02,-2.19349619e-02]],[[7.71381140e-01,-1.00465454e-01,4.49188381e-01],[-3.09704952e-02,6.47085309e-02,-1.06009707e-01],[1.42984045e+00,4.94830102e-01,-1.48505300e-01]],[[2.75627881e-01,3.58115733e-01,-5.89781880e-01],[4.33069170e-01,1.00909688e-01,-3.91711354e-01],[7.29625463e-01,-2.04676419e-01,-1.34420499e-01]],[[-2.87220925e-02,-2.81698078e-01,1.14959389e-01],[-2.32842997e-01,2.08741575e-01,1.12350481e-02],[6.43301308e-01,1.23729251e-01,-2.23495379e-01]],[[-1.86712191e-01,6.34837925e-01,-1.27280146e-01],[-3.64747822e-01,-3.71813178e-02,-5.55383325e-01],[4.58212849e-03,4.26113680e-02,-4.34444070e-01]],[[1.24307501e+00,1.26647905e-01,3.03768903e-01],[1.39266992e+00,8.04565847e-01,-4.11991850e-02],[4.28913027e-01,8.75546858e-02,-8.74478817e-02]],[[-2.15479210e-01,-1.48879707e-01,-4.46119219e-01],[-6.12894952e-01,4.74182516e-01,1.54178455e-01],[-2.00719386e-02,1.21572623e-02,2.10265651e-01]]],[[[-1.80174637e+00,6.75463140e-01,-3.50766450e-01],[-3.62642169e-01,-3.08734238e-01,-1.45874307e-01],[-2.67687708e-01,-3.00883472e-01,6.17262498e-02]],[[1.00636518e+00,5.06808236e-02,-1.36081994e-01],[1.04214692e+00,-4.22661275e-01,2.37147853e-01],[1.19809739e-01,7.55314231e-02,-5.05485237e-01]],[[-7.29003966e-01,6.18693829e-01,1.02506471e+00],[-1.24104194e-01,-2.42158890e-01,-2.42687672e-01],[5.61234593e-01,-3.18635613e-01,1.76265150e-01]],[[-1.97661981e-01,-2.40093082e-01,1.70956254e-02],[2.17577264e-01,-1.29812762e-01,1.51177928e-01],[2.57728159e-01,-5.24609566e-01,-1.60941109e-01]],[[-3.18823904e-01,-3.85105699e-01,-1.80532373e-02],[-5.44412918e-02,-4.72220778e-01,4.13913429e-01],[1.26585409e-01,-3.69870067e-02,-1.79906860e-02]],[[3.76374945e-02,5.26183657e-02,-7.93289021e-02],[-2.26526216e-01,8.79624307e-01,-3.84531692e-02],[6.80619717e-01,-8.27709973e-01,8.66016652e-03]],[[-3.52220833e-02,3.05100203e-01,8.93160999e-02],[4.80862975e-01,-9.02588386e-03,9.02359281e-03],[6.40683770e-02,3.95761169e-02,4.08378392e-01]],[[4.83127028e-01,4.51356657e-02,5.44145942e-01],[5.25419056e-01,-6.21066213e-01,-7.83935711e-02],[-6.65639400e-01,4.46231067e-01,-2.15393245e-01]],[[-6.20188951e-01,2.54682362e-01,4.53324765e-01],[-6.64484799e-01,-1.52731642e-01,5.10215759e-01],[-5.09708166e-01,-8.15061390e-01,-4.77550715e-01]],[[9.97660816e-01,-4.50031869e-02,5.60235158e-02],[7.00153351e-01,-4.74592388e-01,-8.10237586e-01],[4.40274656e-01,-7.83785999e-01,4.31575567e-01]],[[-1.88660979e-01,-6.80820048e-02,-8.63224342e-02],[1.03477383e+00,-3.11751992e-01,-8.42274353e-02],[-9.80168507e-02,-5.59401691e-01,-6.17933393e-01]],[[-7.62491822e-02,-1.33613646e-01,1.55033804e-02],[4.99014735e-01,-1.53123870e-01,4.54931557e-01],[7.90830612e-01,5.81751287e-01,-5.61687529e-01]],[[-2.75807649e-01,-1.58743754e-01,-9.51535761e-01],[-2.06334248e-01,1.54742571e-02,-9.74269688e-01],[-4.20121670e-01,4.16549146e-01,-2.86239475e-01]],[[-1.46091357e-01,-3.80486369e-01,3.08494836e-01],[9.70555484e-01,-5.38723528e-01,-5.00315912e-02],[-3.54165584e-01,-8.34319592e-01,-7.40966856e-01]],[[-4.55901235e-01,1.61849231e-01,-8.47633630e-02],[-1.13657331e+00,-1.59254104e-01,4.24317382e-02],[5.92131317e-01,-2.43765876e-01,-5.46113700e-02]],[[1.98871434e-01,-4.90360320e-01,1.84126586e-01],[9.19893011e-02,-1.72748655e-01,6.59642160e-01],[-1.29051197e+00,-4.05060858e-01,3.22528034e-02]],[[8.02677311e-03,-9.72891748e-02,2.49143705e-01],[-5.16985238e-01,-2.41730124e-01,3.14613879e-01],[3.01881850e-01,-6.35476410e-01,1.70320913e-01]],[[7.54561350e-02,-9.97844618e-04,-7.80790448e-01],[5.10619164e-01,3.75573640e-03,-2.31461599e-01],[-5.44826269e-01,-1.46071061e-01,-8.16681888e-03]],[[-1.72162041e-01,-3.17894332e-02,-7.14591384e-01],[-4.62088853e-01,-6.19392931e-01,-1.48466349e-01],[-7.11804330e-01,9.32572596e-03,-2.82862008e-01]],[[9.87760816e-03,-3.63044679e-01,3.66862863e-01],[5.37540257e-01,3.11083764e-01,-6.40831769e-01],[-1.72759891e-01,9.20598060e-02,-4.17446941e-01]],[[8.75540078e-01,3.67723405e-01,1.03088903e+00],[-4.39011812e-01,-3.56506079e-01,4.31739949e-02],[6.37477487e-02,1.64748862e-01,5.13412245e-02]],[[-2.11220360e+00,-3.39852393e-01,7.61083290e-02],[2.93248355e-01,-1.32842973e-01,-2.37001814e-02],[-2.60341734e-01,-2.09440202e-01,-1.52761012e-01]],[[4.51328866e-02,3.21397334e-01,4.37130392e-01],[-3.84006411e-01,1.91736177e-01,-1.29200011e-01],[-2.28632748e-01,2.15896636e-01,3.97310346e-01]],[[1.80449352e-01,3.68264705e-01,-4.69118685e-01],[-1.47906290e-02,8.53922129e-01,3.24485935e-02],[8.03438187e-01,-2.22188830e-01,8.80283177e-01]],[[2.31276155e-01,-1.11477245e-02,-4.19785976e-01],[4.67886366e-02,-3.23604941e-01,-1.32936627e-01],[5.09903669e-01,1.62905201e-01,-4.28171456e-01]],[[-8.71366680e-01,-6.69374228e-01,3.07759136e-01],[-2.48915568e-01,1.44953251e-01,-2.40687355e-01],[5.25148213e-01,-3.49624217e-01,-6.13254845e-01]],[[2.27226898e-01,1.81589305e-01,3.20324838e-01],[-1.03362091e-01,-3.15976799e-01,1.18628776e+00],[-4.86899048e-01,-2.97072172e-01,-9.38835144e-02]],[[-7.05938518e-01,-1.66204125e-01,-6.09958023e-02],[3.25476825e-01,9.76566449e-02,-4.96906340e-01],[-8.30212533e-01,2.88177520e-01,-2.66465664e-01]],[[-3.52757305e-01,-6.23448193e-01,-6.03152752e-01],[-4.57676798e-01,-1.94176465e-01,-1.97796553e-01],[-3.08927596e-01,6.74275830e-02,4.76178080e-01]],[[2.54246920e-01,2.27280810e-01,2.87504625e-02],[1.96894214e-01,-5.09761944e-02,5.94418459e-02],[1.07843876e-01,-5.12319922e-01,-9.60933089e-01]],[[1.43227470e+00,3.10978621e-01,-6.07391655e-01],[-2.06355572e-01,-1.20146319e-01,-2.30290428e-01],[4.25383687e-01,-2.73361236e-01,-1.56092092e-01]],[[4.35579658e-01,1.11170483e+00,7.71667957e-01],[3.65263402e-01,1.29685700e-01,2.07338169e-01],[-9.85776067e-01,-4.50198501e-01,1.09637387e-01]],[[7.50981927e-01,1.34416163e-01,-5.71284413e-01],[1.29448247e+00,3.64396907e-02,-4.70662594e-01],[-1.91892812e-03,-3.06668580e-01,-5.62859774e-01]],[[-2.85770983e-01,-2.27919415e-01,2.57781476e-01],[4.66841489e-01,9.46883410e-02,3.19618255e-01],[2.57663310e-01,-2.14158654e-01,-4.32280265e-02]],[[4.00400728e-01,-1.72114372e-01,3.95038933e-01],[2.76019335e-01,4.42905009e-01,2.40045398e-01],[2.19246998e-01,6.23525321e-01,-1.13537394e-01]],[[3.81979167e-01,8.37563515e-01,-3.89894068e-01],[2.52563179e-01,-4.55372691e-01,-1.74984023e-01],[-9.35063362e-01,-2.45683804e-01,-1.89611316e-01]],[[9.30037275e-02,1.63893849e-01,-7.09931314e-01],[-3.14279161e-02,-7.67519772e-02,-3.33081931e-02],[3.36642623e-01,-9.20760408e-02,-6.88517168e-02]],[[-4.26294684e-01,1.81356236e-01,7.97590375e-01],[5.45240700e-01,4.75969404e-01,-4.58738916e-02],[-2.98110992e-01,3.44582081e-01,-1.87911689e-01]],[[2.77654529e-01,-1.28101319e-01,-1.76408906e-02],[4.59903449e-01,-2.28720888e-01,1.06396639e+00],[-1.21090877e+00,4.93010968e-01,5.98271750e-02]],[[4.32258159e-01,-2.94606328e-01,1.35665476e-01],[4.66511488e-01,1.34146675e-01,-1.51877388e-01],[8.10426116e-01,-1.29081801e-01,4.78725702e-01]]],[[[-1.364696,-0.54767364,0.5874834],[-0.15354301,-0.51765263,0.4795065],[-0.04379075,0.01298684,-0.09150438]],[[-0.27748242,0.38070887,0.00460506],[0.1722479,-0.06137976,-0.05560431],[0.19591886,0.5580616,0.54198354]],[[-0.24809213,0.49276826,1.0200882],[-0.31738153,-0.04934435,-0.2850557],[-0.55869704,0.6807859,0.7460535]],[[0.03091965,-0.2026619,-0.40126356],[-1.1167436,-0.24021968,-0.10203068],[-0.27352214,-0.18223877,-0.4194382]],[[-0.37847966,-0.39260757,-0.5110349],[0.4729534,-0.32753947,0.23362337],[-0.14263688,0.41703385,-0.3745189]],[[0.06379531,0.13746765,1.065544],[-0.3267589,0.4636226,-0.3404677],[0.00809598,-0.40458965,0.19912653]],[[0.45589465,0.59139943,-0.6107571],[-0.39420277,0.08135733,-0.52568156],[0.35511,-1.511255,-0.44102305]],[[0.31051454,0.19682695,0.616225],[0.18041292,0.06174987,0.32974035],[-0.7674299,0.3681447,0.4332202]],[[-0.2068878,-0.15537596,0.07883225],[-0.0500249,0.40706685,0.47630405],[-0.25091708,-0.28430486,-0.43799898]],[[1.114864,-0.02005183,0.4015081],[1.3420258,-0.16705063,-0.19212314],[-0.05148772,-0.58853036,-1.0323116]],[[-0.42711318,0.24082159,0.09305283],[-0.26180193,-0.40219003,0.49141982],[0.67091465,0.85705286,-0.14073622]],[[-0.7552226,-0.04598201,-0.11766543],[-0.15796563,0.32073575,0.6229355],[-0.4436072,-0.24152581,0.80484265]],[[0.554071,0.1905219,0.5664299],[0.38040408,-0.24755463,0.22480257],[0.28236765,0.5238946,0.14667054]],[[0.6057269,-0.10520292,0.21804446],[-0.45199203,0.29657057,0.15593624],[0.65076244,-0.46174386,-1.6017584]],[[0.06977355,-0.6764028,-0.19245751],[0.0290148,0.09476341,-0.46866205],[-1.230843,0.40472272,-0.6069731]],[[0.4970687,-0.22571136,-0.50065523],[0.08505937,-0.35441396,-0.37012473],[0.8192669,-0.4150597,0.17432262]],[[-0.13891143,-0.20565711,0.03127201],[0.38768137,-0.06332666,0.10921813],[1.0310067,0.3386972,1.1025784]],[[-0.04377951,-0.0392457,-0.07852663],[-0.5564773,0.07623433,0.4432736],[0.49381065,-0.46570536,0.02843188]],[[0.00956891,-0.21436642,-0.44855627],[-0.6737531,-0.33565354,0.3230806],[0.11836898,-0.29787028,-0.76832986]],[[0.01718887,-0.3480503,-0.03981359],[-0.06402482,-0.01181634,-0.7725392],[-0.00571558,-0.07177977,0.01002732]],[[0.35745955,0.28746527,0.47623134],[0.41787216,-0.3340587,0.07864282],[-0.09780003,-0.07457775,0.22365116]],[[-0.21629103,-0.03698976,0.26559728],[-0.32982987,0.05838567,0.22109872],[0.07536557,-0.03601691,0.24172567]],[[-0.21801288,-0.26338175,-0.50589985],[0.54378825,0.16498308,-0.42958403],[-0.13258517,-0.10229395,-0.25680956]],[[-0.32006738,0.1414557,0.05543908],[-0.31535545,-0.05434163,-0.06855771],[-0.02310925,0.1899497,-0.24849398]],[[-0.29401806,0.19728054,0.08813132],[-0.12445766,0.21102902,-0.38427302],[0.0338947,-0.02336791,0.5238651]],[[0.02847185,0.27448988,-0.16985326],[0.0711055,0.7716225,0.4008732],[-0.24275853,0.59103197,0.58973813]],[[-0.7702627,0.17474385,-0.11242367],[-0.2467246,-0.11063952,0.07436027],[0.652769,0.20092003,-0.45007622]],[[0.51903945,0.15854456,0.3644144],[0.47946885,-0.37594074,-0.80900663],[1.0021799,0.09222317,-0.20223954]],[[0.17076157,0.5762405,-0.07872551],[0.24536374,-0.58001506,-0.15783346],[0.00477341,-0.0670148,-0.32681078]],[[0.3857473,-0.44945955,0.14697237],[-0.33148503,0.15962887,0.23572928],[-0.32793733,-0.8199712,-0.16662346]],[[-0.38642684,0.3545069,-0.10013362],[-0.50491524,-0.18214038,-1.025299],[0.23613472,0.09511405,0.10669609]],[[-0.0419989,0.18900776,-0.12480196],[0.20015225,-0.4575743,-0.6639622],[-0.7026542,0.34431592,-0.2858307]],[[-0.9392013,-0.2671147,-0.24446684],[-0.04834378,0.5253025,0.31928998],[-0.25572377,-0.38157216,0.03717303]],[[-0.6080663,0.08867607,0.27490592],[0.29370478,-1.1616209,0.20878725],[0.32797065,-0.06273364,0.69554204]],[[0.7943206,-0.2710034,-0.42674294],[0.22277504,0.3488437,0.6566126],[-0.2857333,-0.8456008,0.25025278]],[[-0.6233892,0.22915453,0.5043007],[-0.04890789,-0.7430978,0.09733396],[0.57067484,-1.3301411,-0.67792153]],[[0.72429234,0.2983239,-0.21989238],[-0.04921024,0.14174509,0.00416575],[-0.22689812,-0.338395,0.960682]],[[0.20931208,-0.62387073,0.5763263],[0.67794555,0.47192585,0.18661267],[-0.27404627,-0.6288106,0.19442013]],[[-0.36859745,-0.36674377,-0.5021935],[-0.27227148,0.3049114,-0.8018779],[0.1557713,-0.02518311,-0.11131197]],[[-0.2902939,0.10684458,0.05664474],[0.6076559,0.05570885,-0.56609213],[0.4388242,-0.09552943,-0.31578428]]],[[[0.61579376,0.23118983,-0.19486116],[0.1446413,0.10322452,0.40415475],[0.04393172,-0.16666944,-0.2928498]],[[-0.16666037,-0.70135206,-1.3197346],[0.13300101,-0.20404531,-0.7590629],[-0.4817445,0.28696972,-0.14416444]],[[0.07350092,-0.7473709,0.09744021],[-0.30351552,-0.16670357,0.1141093],[0.18590035,0.23302796,-0.2460324]],[[-0.11163632,0.79788476,-0.25003996],[0.04909895,0.20365974,-0.36728975],[0.0339269,0.33516708,0.01632919]],[[-0.04348254,0.01019177,-0.3614399],[0.10806667,0.18665192,-0.422072],[-0.10274497,0.2860984,0.30728692]],[[0.4896553,-0.47083294,-0.36440694],[-0.37804106,-0.12745374,0.24577567],[-0.16108675,-0.6959641,0.04088276]],[[-0.3023858,0.4646944,0.7738798],[0.3460902,-0.5633343,0.04835786],[0.05490359,-0.08389539,0.7046266]],[[-0.11060815,0.17959599,-0.69668156],[-0.23587075,0.05591571,-0.08749215],[-0.96957123,-0.82155067,-0.08094151]],[[-0.65259886,0.2964604,-0.72155446],[0.6057394,0.30922472,-0.10127806],[0.3050359,-0.820565,-0.21982895]],[[-0.65508646,0.40251616,-0.6179217],[0.3254054,0.49363938,-1.2449348],[-0.7486413,-0.4413689,0.05607627]],[[0.29286674,0.7711614,0.2285883],[-0.25635025,-0.4830213,0.6199472],[0.50333476,0.01127328,-0.26642406]],[[-0.23553042,-0.17864282,-0.28545612],[-0.31701997,-0.09125666,0.4307598],[-0.19854593,-0.04295801,0.5297124]],[[-0.06605197,-0.04126953,-0.1472398],[-0.01450614,0.67013174,0.43765885],[-0.5136778,0.48325846,1.2206513]],[[0.6213523,-0.17864844,-0.23943248],[0.12587243,-0.8499297,-0.50921637],[0.87667775,0.33575773,0.08032607]],[[0.15043415,-0.37611502,0.6326914],[0.11658026,-0.61091906,1.1990484],[-0.7814402,0.19854291,-0.55714774]],[[-0.30243668,1.1025354,-0.13865902],[0.07165175,0.4575752,-0.18178304],[-0.22125135,-0.56849194,-0.68354565]],[[0.14999066,0.6565341,0.27299985],[0.41388056,0.2279897,0.8563797],[-0.3545993,-0.21631426,-0.344449]],[[-0.7849578,0.59048945,0.21070334],[-0.08394843,-0.4765315,-0.6885332],[0.14948076,-0.37681872,0.01928982]],[[0.44958088,-0.6996275,0.30347684],[-1.0578,-0.91237205,-0.5680234],[0.9121371,-0.29319218,-0.59610146]],[[-0.17107645,0.12589274,0.44858667],[0.55473083,-0.37275532,0.22692066],[-0.403818,-0.7629412,-0.34708276]],[[-0.20707558,-0.65042955,-0.84034526],[0.13223666,0.11827708,-0.30792883],[0.27230984,-0.41653365,-0.7782636]],[[-0.42876458,0.14695373,0.91138566],[-0.525896,0.18435702,0.51695704],[0.38495028,0.19241744,-0.6109301]],[[0.16649665,-0.5585131,-0.05500878],[0.07841542,-0.50828266,0.05778176],[0.90233433,-0.24082454,-0.50950485]],[[0.59320146,0.05866481,0.05558697],[-0.43497851,0.16524139,0.00966876],[0.3838334,0.28427088,0.0138007]],[[0.2606954,0.16350573,-0.16746108],[0.04670396,-0.21072613,0.724669],[-0.14838922,0.279188,-0.11879045]],[[-0.01779087,-0.3141414,-0.16577427],[0.16079724,0.26556247,0.07033983],[-0.6128022,0.56855685,-1.0883033]],[[0.17905566,0.8161806,-0.01277335],[0.6654273,0.23587143,-0.42731467],[0.0813209,-0.27744243,-0.04883846]],[[-0.55757105,0.0663427,1.0546616],[0.15707672,-0.3368304,-0.07455271],[-0.33669,0.1595775,-0.84455717]],[[0.24061641,0.59739774,-0.09454925],[-0.06626242,0.22506887,-0.2765821],[-0.6733988,0.0811505,0.26730737]],[[0.85903704,-0.10664479,0.3784464],[-0.12764995,-0.15563805,0.23231678],[0.14418736,0.1913642,-0.22397049]],[[1.0105019,-0.27596942,-0.5670952],[0.22888878,0.21336019,-0.11766955],[0.77134955,0.5011293,-0.16902797]],[[-0.48053646,-0.10527843,-0.04491616],[0.05141191,0.2062656,-0.03281259],[0.05569324,-0.6816486,-1.1777196]],[[1.114839,0.07902649,-0.21647932],[-0.09297288,0.73194224,0.15260877],[0.189448,-0.5267608,-0.69056076]],[[0.30653128,-0.05225533,-0.6434063],[-0.58304715,-0.3054267,0.08571044],[-0.35461923,0.316659,0.01845461]],[[0.35588956,0.1726833,0.16986273],[-0.2243873,0.81068164,0.08764557],[-0.8449005,0.2751595,-0.31025183]],[[-0.06386231,-0.01624373,0.8682726],[0.15014353,0.319877,-0.16419171],[0.1528316,-0.6016649,0.5221207]],[[1.3669962,0.24144602,-0.38047352],[1.023895,0.1032852,0.340847],[-0.01447504,-0.1087512,0.4066638]],[[-0.50575167,-0.37564248,-0.05993072],[0.27716485,-0.5978429,0.11481581],[-0.6105572,-0.59200984,-0.2001511]],[[0.29056573,-0.13479818,0.79711676],[-0.4948177,-0.12748584,0.11477385],[0.10667138,0.5969902,-0.10839023]],[[-0.22826548,-0.01197194,0.13024017],[0.37774697,-0.7981543,0.46732128],[-0.21301739,0.7130888,-0.56792694]]],[[[-0.5752965,0.19251059,-0.6845727],[-0.06537668,0.6071592,0.13896306],[-0.290562,0.23694438,0.07111514]],[[0.44424295,-0.09739528,-0.1623827],[0.05055295,0.14573772,0.17400084],[0.6044498,0.00649066,0.22466928]],[[-0.06498373,-0.5385703,0.26071182],[0.5537128,-0.6201347,-0.39688686],[-0.25427353,0.10599715,0.10621943]],[[0.47174612,-0.02640633,0.5326814],[0.11425117,0.32302466,0.39038092],[0.45501122,-0.5174634,-1.0245454]],[[-0.18421087,-0.00440184,0.19568913],[-1.0865399,0.12586269,0.03764095],[0.07618821,0.44084877,-0.61424935]],[[-0.36558333,0.23542283,-0.4904079],[0.5332764,-0.1857308,0.393592],[0.82821316,0.30006194,-0.6850554]],[[-0.03167293,-0.65726745,0.37904754],[0.07731984,-0.13050123,-0.07814916],[-0.01359356,0.25777856,-0.08981758]],[[0.53718466,-0.46414307,-0.08766144],[-0.22868313,-0.38913807,0.27082187],[1.2007746,0.23177196,0.33355975]],[[-0.06165201,-0.28935143,0.557729],[-0.16650301,0.6760175,-0.01578585],[-0.00930092,-0.05049084,-0.43162078]],[[0.61341804,-0.43140042,-0.10654186],[0.30882773,-0.47107977,-0.15243006],[0.37691906,0.71251625,0.2308813]],[[0.48070326,-0.42504084,0.8165397],[-0.32574928,-0.38512588,0.38617063],[0.07095596,-0.5324989,1.0636775]],[[0.15578865,-0.14824782,-0.1518594],[-0.57512915,-0.20374428,0.4195343],[0.3906582,0.36056933,-0.42797893]],[[0.350923,0.37291068,-0.34963685],[0.14859135,-0.04352959,-0.49390796],[0.32865033,0.69313246,-0.4896412]],[[0.36523408,-0.6330097,-0.6561867],[-0.92185223,-0.25010917,-0.3211461],[-0.82978815,-0.03356039,-0.00487274]],[[0.2331842,-0.0437066,-0.06610666],[0.60604393,0.6317025,0.18431897],[0.41847837,0.35528412,0.11365826]],[[0.32320753,-0.33116466,0.3713861],[-0.26312667,-0.13854672,0.7876346],[0.4833874,-0.01566788,0.94155854]],[[0.229916,0.14343472,0.34452602],[-0.14728992,-0.2631609,-0.2427031],[0.14462972,-0.27220073,-0.23753269]],[[-0.28483307,0.41713387,0.26162216],[-0.17554572,0.05876921,-0.3955177],[-1.0634735,0.7541472,-0.63401717]],[[0.12147375,-0.501009,0.21510977],[0.01064445,-0.48603344,-0.23494096],[0.73778427,-0.29548427,-0.2984051]],[[-0.05855019,0.11813813,0.63659805],[-0.24108139,0.39329383,-0.30455247],[0.50366974,-0.6239988,0.01029365]],[[0.04168725,-0.5671449,-0.8137293],[0.33853957,-0.38711867,0.8994567],[-0.53010523,-0.2520936,-0.5610316]],[[-0.04309074,0.47396788,0.03956602],[-0.30735162,-0.2821109,-0.05621049],[-0.02331125,-0.11011133,-0.2038668]],[[-0.28707635,-0.34329602,-0.42971033],[-0.10611895,0.35206226,-0.40664676],[0.0328589,0.16736013,0.01151113]],[[-0.32847124,0.22519436,0.27865577],[-0.03855936,-0.27401584,0.1562483],[-0.6233518,-0.00458593,0.19239315]],[[0.09184064,0.29255095,-0.11779838],[0.13557631,0.36957082,-0.52314574],[0.46169397,0.28881642,0.19141889]],[[-0.3634502,0.2913612,0.22434774],[-0.45483577,-0.44974047,0.21354066],[0.06152073,-1.2519414,0.15186556]],[[-0.21983062,0.33250758,-0.0309515],[0.22061825,-0.10090938,-0.69667137],[0.18736836,0.44077832,0.02675726]],[[0.2327121,0.29341376,0.6647969],[-0.45755365,0.6392184,-0.07637117],[-0.6611814,-0.6017842,0.09664989]],[[-0.34361234,-0.30277142,0.02635289],[0.04070799,0.32058132,0.7405642],[0.26393685,0.654621,0.6839493]],[[-0.15259251,-0.21864268,0.14153856],[-1.5889951,-0.40328053,0.09235806],[-0.9520099,-0.07490183,-0.03303661]],[[0.3614876,-0.25215,0.40148336],[0.8180166,-0.19825532,0.18397176],[0.10478161,-0.15808703,0.18163425]],[[-0.6526066,-0.06616122,-0.21140322],[-1.6501582,0.25582263,0.29082513],[-1.496205,0.511442,0.80526716]],[[0.14712119,0.03207306,-1.4035648],[-0.00217468,-0.5283494,0.16618046],[0.41757885,0.76355195,-0.12136657]],[[0.5455096,0.18809886,0.4435442],[0.77106184,-0.00731558,0.13857609],[-0.06584308,-0.1349528,0.5368049]],[[-0.47061053,-0.79487884,0.2641361],[-0.11874938,-0.07432238,0.14749464],[-0.133504,-0.09825125,0.21216068]],[[0.44158086,-0.24237747,0.08905733],[0.01875673,-0.87942904,-0.4366971],[-0.35095152,0.02147462,0.33017448]],[[-0.6928575,0.05810879,-0.63918096],[-1.2102848,-0.39792573,-0.682479],[0.0718335,-0.10157181,0.53159124]],[[0.19208093,0.06982926,-0.7699604],[-0.32070693,0.36290723,-0.6276867],[0.19555654,-0.00316085,-0.7011467]],[[-0.5595513,0.6696717,0.47356],[0.05422395,0.11304279,0.27883634],[-0.6099692,0.4821698,-0.16545412]],[[0.67596143,0.03928418,0.27182546],[1.4486674,-0.2325585,0.1153708],[0.70159066,-0.40462264,-0.42961258]]],[[[-0.67623407,-0.07509325,-0.3020641],[0.06750609,0.3278427,0.04997228],[0.19812469,0.38432726,0.10522811]],[[-0.22922704,-0.24388905,-0.54836667],[0.50223666,-0.49209037,-0.21291439],[0.6646114,0.17438881,0.31082392]],[[-0.07403175,1.0170822,-2.592564],[-0.42566025,0.8127351,0.05987602],[-0.2570702,0.55193484,-0.24503438]],[[0.16067216,0.16973196,-0.11580823],[0.31755865,-0.05811015,-0.36077452],[0.01272455,0.10962865,-0.32935527]],[[-0.6334555,0.12193809,0.3388683],[-0.43628427,-0.7809355,-0.14449233],[-0.37141353,-0.6251039,-0.36764103]],[[0.6012372,0.60093,0.0247265],[-0.26800856,0.16725302,-0.84555614],[-0.33561537,-0.09271544,-0.16825917]],[[1.0807126,0.30931515,-1.0902758],[0.3112324,0.0759168,-0.3649481],[-0.16959818,-0.44111,-0.5981887]],[[-0.05307706,0.30979326,0.27054003],[-0.12316615,0.35800445,-0.12413792],[-0.3337345,-0.28613243,-0.5136476]],[[-0.11079793,-0.13182147,0.50850284],[-0.2633109,-0.41881943,0.61625546],[0.10035139,-0.6060859,0.05218776]],[[0.65455484,0.22287278,-0.17564368],[0.4964826,0.3470583,-0.4127793],[0.10529953,-0.21314314,0.5697883]],[[0.16470976,0.43983456,0.31112564],[-0.06648245,-1.4108492,0.4495735],[0.3143572,0.08295418,0.5861284]],[[-0.2873225,0.5669706,-0.6019473],[-0.3562644,0.11058522,0.33697897],[-0.5512739,0.0434637,-1.0510728]],[[0.961852,-0.20553604,-0.29138234],[0.17983827,-0.20736179,0.32700813],[-0.8136553,-1.0025978,-0.4173646]],[[-0.05527627,0.13453901,0.9315272],[0.24407257,-0.8892369,0.42962244],[-0.03610215,0.08880402,-0.06732573]],[[-0.46980432,0.04393583,0.54066086],[-0.0463176,-0.04576379,-0.48366988],[-0.08425803,-0.12327181,0.26524982]],[[0.56755227,-0.6808084,1.0886097],[-0.83722043,-0.86757547,0.45642307],[-1.1602582,-0.7545866,0.44077778]],[[-0.5989734,-0.05466339,-0.23240924],[0.38688317,-0.58013844,-0.05048823],[0.5681873,0.26005,-0.3954312]],[[-0.67826426,-0.89993036,0.4529166],[0.9495449,-0.27958357,0.16353282],[-0.8521944,-0.55960125,0.9173775]],[[0.7649994,1.0129343,-0.03546251],[-0.0475535,-0.24495153,-0.18308885],[0.11454091,-0.38915718,0.32389945]],[[-0.11040124,-0.5082209,0.6384131],[0.25823155,-0.06165675,-0.46057943],[-0.00507058,0.856462,0.04232437]],[[0.11779821,0.06976207,0.5630672],[0.28567916,0.2810999,0.27767295],[-0.09787189,0.02006317,0.46829942]],[[0.8766239,0.03062216,-0.07307497],[0.09992895,-0.4055547,0.07053606],[-0.364015,-0.4852275,-0.13403991]],[[0.09091251,0.17955329,-0.22413284],[0.40442154,-0.09785936,-0.17332512],[-0.7243217,-0.03685387,-0.16191901]],[[0.29549804,0.02812887,-0.1546],[-0.22995177,-0.4202353,-0.42948532],[-0.6249522,0.13091265,-0.1592431]],[[0.27481905,0.1361395,0.05197095],[0.60922664,-0.5879106,-0.44267735],[-0.27374312,-0.01340534,0.46717796]],[[0.25931972,0.20406209,0.13577542],[0.29178926,0.386215,0.33164808],[-0.36221322,0.47703952,0.59131247]],[[-0.13789062,0.19761583,0.6943952],[-0.35550347,0.19348541,-0.18496022],[-0.5605566,0.26621196,-0.15602261]],[[0.01954624,-0.74697787,0.08765049],[0.2515726,0.1630339,-0.4214113],[-0.23026432,0.6222667,0.09738491]],[[0.32283846,-1.7540679,0.6001194],[-0.05016347,-0.449761,0.27764276],[0.04936033,0.08032403,0.531285]],[[-0.48573196,0.14711756,-0.41454306],[-0.11180468,0.07381127,-0.42114055],[-0.39249334,0.33461523,-0.24174309]],[[0.8710898,-0.8058795,0.5670868],[0.79138607,0.08582961,-0.704881],[-0.14507918,-0.21226694,-0.20009302]],[[0.35590628,0.5987592,-1.4250952],[-0.37202072,0.05058088,0.38731351],[0.38149092,0.3832315,0.36089236]],[[0.161926,-0.37733874,0.46948394],[-0.18141142,-0.15893121,-0.24484137],[-0.23102759,-0.36591196,0.21476012]],[[-0.67614496,-0.0121598,0.53749853],[-0.12047885,0.43770066,0.4604862],[0.3220232,0.5701577,-0.13949436]],[[-0.14716104,0.02790497,0.16693492],[-0.5405586,0.2449147,-0.2657249],[-0.13455366,0.00783437,-0.22903064]],[[0.30316126,0.45472488,0.39058033],[0.40695056,-0.4618042,0.01869799],[0.637033,-0.29528207,0.49214306]],[[-0.5263321,-0.5390045,-0.4813407],[0.29305565,-0.35992795,0.02982464],[-0.73139375,-0.21294986,-0.2635104]],[[0.16378912,-1.1012782,0.32907143],[0.30153364,0.2063551,0.33248153],[0.216765,-0.7281527,0.37760764]],[[-0.515638,0.18647653,0.08628508],[-0.28804895,0.17073727,-0.10119918],[-0.58127296,0.45937753,0.45066726]],[[-0.42894807,0.99505085,0.19005877],[-0.6735579,0.10996184,-0.16834734],[-0.04415524,0.06170579,-0.393004]]],[[[-4.74602669e-01,6.24209464e-01,-5.04496992e-01],[-4.48154181e-01,-1.35458615e-02,-2.04682909e-02],[-1.28528967e-01,1.52372301e-01,-2.11959466e-01]],[[2.59809971e-01,2.53682375e-01,5.46491027e-01],[-3.21872354e-01,-3.05231541e-01,-2.19735596e-02],[1.06677163e+00,2.64560759e-01,5.15349865e-01]],[[-2.38521069e-01,-1.80990398e-01,-3.26926142e-01],[5.75716555e-01,3.07795733e-01,-3.63161027e-01],[4.45611835e-01,1.28706517e-02,3.88389081e-01]],[[3.11067075e-01,-8.05748776e-02,3.10803741e-01],[-2.58298758e-02,1.42085969e-01,-3.33591193e-01],[-2.90911585e-01,-2.85936415e-01,4.42007929e-02]],[[7.77764142e-01,2.96364486e-01,2.75875419e-01],[2.67199636e-01,-5.18300105e-03,7.23352283e-02],[5.34938633e-01,1.97880641e-01,2.24705279e-01]],[[-2.05087855e-01,4.02620107e-01,-5.70610702e-01],[-6.40872002e-01,3.27274233e-01,2.31699720e-01],[-1.53848574e-01,3.35835010e-01,-4.70317334e-01]],[[1.04124391e+00,-1.93514779e-01,3.51517409e-01],[-6.34520724e-02,1.47178248e-01,-3.25818390e-01],[7.16541946e-01,-7.25491270e-02,8.75723839e-01]],[[-9.00003687e-03,-4.74520713e-01,-9.61512551e-02],[2.98631281e-01,-3.48386616e-01,2.58861035e-01],[-1.35502622e-01,-5.88298738e-01,-1.79006711e-01]],[[3.49962786e-02,2.94859499e-01,-8.56390953e-01],[-2.31946066e-01,-4.51331466e-01,-1.14717729e-01],[3.89760323e-02,-6.61412776e-01,-6.23559117e-01]],[[2.77867049e-01,5.57488322e-01,3.05387229e-01],[-4.77875978e-01,-1.27926961e-01,6.29856288e-02],[-6.20105445e-01,-3.56822580e-01,1.98036313e-01]],[[-7.11896792e-02,1.21664554e-02,-2.65420228e-01],[5.66062450e-01,3.74720199e-03,-7.91140869e-02],[-3.13906699e-01,1.57543018e-01,1.03938878e+00]],[[-8.37964416e-02,-8.57129753e-01,6.98087886e-02],[4.11601692e-01,3.78883064e-01,1.31312981e-02],[-9.21262622e-01,3.23913395e-01,-5.13178259e-02]],[[4.89639118e-02,-6.65599942e-01,7.63157010e-01],[-7.85587609e-01,-2.19632372e-01,8.64914805e-02],[2.37181649e-01,-1.58602223e-01,-6.64259255e-01]],[[2.79366136e-01,-6.59133568e-02,-6.13819174e-02],[-3.11393589e-01,2.00666010e-01,3.95247638e-01],[8.04148316e-01,-6.03171401e-02,8.14960361e-01]],[[-1.22061603e-01,3.11065882e-01,-3.29839826e-01],[-1.25681628e-02,1.69188589e-01,6.00109875e-01],[6.28582835e-01,4.42886412e-01,6.14326000e-01]],[[-2.42333021e-02,3.10912222e-01,1.46992430e-01],[3.19777042e-01,3.11158411e-02,-3.56195793e-02],[6.91768706e-01,-6.78717911e-01,1.77540350e+00]],[[8.25024117e-03,-4.67903763e-01,-6.03807092e-01],[-6.85343146e-02,-3.74182701e-01,2.43427217e-01],[-7.29029000e-01,-7.75663704e-02,1.14555687e-01]],[[2.38269344e-01,-1.76332474e-01,2.40378037e-01],[-1.00253808e+00,3.97617966e-01,-2.13662967e-01],[7.20251873e-02,4.89244401e-01,6.08622611e-01]],[[5.40364832e-02,-4.16554153e-01,7.55933762e-01],[2.59425282e-01,-2.91797042e-01,2.65666842e-01],[3.17280293e-01,-2.93838888e-01,-7.47173190e-01]],[[3.46542776e-01,-2.93310672e-01,6.88244253e-02],[6.13423944e-01,2.05207646e-01,-1.35795638e-01],[-9.19940054e-01,-1.46233626e-02,-8.66264760e-01]],[[6.53879702e-01,-6.72019958e-01,7.61495054e-01],[-6.52843118e-01,5.03355443e-01,3.01234275e-01],[6.68982208e-01,4.65064347e-01,-3.08845907e-01]],[[-1.59084511e+00,-3.20712835e-01,1.48288190e-01],[-8.60243797e-01,-5.93536675e-01,6.63512424e-02],[-6.16617322e-01,-3.34531069e-02,2.16924101e-01]],[[-6.14979029e-01,-3.31889838e-01,-3.39698404e-01],[4.60574031e-01,-6.91646710e-02,-6.26828790e-01],[2.28493318e-01,6.08971477e-01,3.16356830e-02]],[[-2.37275064e-01,1.61562320e-02,1.01567924e+00],[-4.92157280e-01,8.16866577e-01,3.18476677e-01],[3.12638074e-01,8.79868805e-01,1.66035786e-01]],[[-7.08047926e-01,-4.68349814e-01,2.28381127e-01],[-2.17465311e-01,4.58343983e-01,-8.25428963e-02],[-4.91762049e-02,2.44983926e-01,-7.47293353e-01]],[[-1.63833320e-01,9.08111453e-01,-4.21099328e-02],[1.51447535e-01,6.92579865e-01,-4.38178241e-01],[-3.90543193e-01,-1.76027447e-01,1.94216177e-01]],[[2.99783677e-01,-8.38601828e-01,3.89357388e-01],[-4.10343111e-01,9.23748851e-01,-2.55149722e-01],[4.87327874e-02,-6.21091843e-01,-1.09885848e+00]],[[6.05389066e-02,1.23566976e-02,-2.82324582e-01],[6.42412484e-01,6.01556301e-01,7.61703968e-01],[2.76505679e-01,5.72881699e-01,3.57061327e-01]],[[-6.19351678e-02,3.24728906e-01,1.18751943e+00],[2.89032966e-01,-4.17658776e-01,1.83325663e-01],[3.14322829e-01,3.77261862e-02,5.80358803e-01]],[[-1.57914191e-01,-3.07347655e-01,-6.35904789e-01],[-7.38759115e-02,-2.09918573e-01,-4.13041234e-01],[9.34821188e-01,-5.07181168e-01,-1.34206924e-03]],[[-2.34742731e-01,-3.32581401e-01,5.90104520e-01],[-3.61615159e-02,-1.47408396e-01,5.81162333e-01],[-7.11506829e-02,5.68946719e-01,-2.20914762e-02]],[[6.84260190e-01,4.14270908e-01,-4.78806138e-01],[6.04174495e-01,-3.84287477e-01,2.76725322e-01],[3.86862278e-01,-1.00837730e-01,6.59952939e-01]],[[2.97272950e-01,3.21104825e-01,6.42576441e-02],[4.03315783e-01,-9.16992247e-01,6.08693123e-01],[7.17510879e-02,4.57850426e-01,8.67892429e-02]],[[-2.19927773e-01,-1.84568018e-01,2.77885944e-01],[2.75277533e-02,2.99063116e-01,5.18302083e-01],[-3.36098790e-01,9.13210958e-02,2.05323890e-01]],[[-2.08603621e-01,-1.08494192e-01,3.44923556e-01],[-7.76484132e-01,6.89117685e-02,7.50734746e-01],[-1.22695893e-01,3.03840548e-01,9.01494622e-01]],[[2.39094436e-01,-5.30666590e-01,3.87851328e-01],[-5.83008051e-01,3.70922327e-01,-5.47497630e-01],[-3.22262108e-01,6.32965624e-01,-7.51449645e-01]],[[-6.43631935e-01,-2.23101825e-01,-1.15432061e-01],[-7.73974136e-02,-1.50043860e-01,-5.56632519e-01],[-6.67356312e-01,1.28937298e-02,-6.20023727e-01]],[[-6.40728712e-01,-2.19430223e-01,1.19574510e-01],[-1.30175114e-01,-4.93736304e-02,2.73107916e-01],[-1.86585143e-01,-1.49776682e-01,9.27390754e-02]],[[-7.15820551e-01,2.54617840e-01,-1.86352491e-01],[7.59095371e-01,-1.22268602e-01,-4.34156150e-01],[-2.55994350e-01,5.45321330e-02,1.23793297e-01]],[[-1.19220115e-01,-7.00922072e-01,5.81254840e-01],[3.25446606e-01,-4.45213020e-02,3.56211215e-01],[-1.98258340e-01,1.22758545e-01,-2.81184703e-01]]],[[[-5.29823303e-01,5.16091228e-01,2.79240161e-01],[-5.72819412e-02,2.66088933e-01,-3.31672907e-01],[2.53382057e-01,6.64073527e-02,3.62494588e-01]],[[-1.56483144e-01,-1.00749590e-01,-2.96147317e-01],[-1.32496148e-01,-9.53833163e-01,1.17634642e+00],[1.30740002e-01,-3.82166654e-01,-2.52223164e-01]],[[4.11568642e-01,-1.66089311e-01,-1.57039285e-01],[-2.16508165e-01,-3.81192803e-01,3.25524896e-01],[-5.63052222e-02,-3.27793509e-01,-8.84340778e-02]],[[3.31870198e-01,1.85458928e-01,-6.51881099e-01],[-1.46679029e-01,8.33184570e-02,4.47949111e-01],[1.12950899e-01,5.12716591e-01,9.64607075e-02]],[[5.18199690e-02,-1.89744860e-01,-4.36616927e-01],[5.62287986e-01,-4.28929448e-01,-1.21176019e-01],[9.31010902e-01,-6.70970738e-01,-2.27522571e-02]],[[4.92566288e-01,-1.70946836e-01,-1.16811462e-01],[-4.02971506e-01,3.62716019e-01,1.91128969e-01],[4.15564567e-01,5.20723999e-01,1.91920221e-01]],[[-3.55392881e-02,2.56847087e-02,5.79289913e-01],[-9.54483543e-03,-4.14757311e-01,8.48105252e-01],[3.32729220e-01,5.79344988e-01,-5.81767643e-04]],[[3.95705290e-02,-1.15811720e-01,6.66474879e-01],[-4.16107625e-02,3.03520977e-01,-3.47137988e-01],[-1.98722109e-01,-2.33353943e-01,-8.18499088e-01]],[[-1.39712512e-01,-5.09793460e-01,2.93842643e-01],[-5.05543172e-01,-4.72570151e-01,-3.85833740e-01],[3.28244060e-01,7.88483620e-02,-3.76597047e-01]],[[-8.27290654e-01,3.14348459e-01,1.58577234e-01],[5.76504230e-01,-2.34619036e-01,-4.75669503e-01],[3.38324368e-01,-3.34424712e-02,9.46501046e-02]],[[-4.64453667e-01,3.70119065e-01,1.02682638e+00],[3.82502496e-01,7.26251721e-01,2.69792467e-01],[-4.56318170e-01,-2.06276745e-01,-5.98053634e-01]],[[-6.43262491e-02,-2.02113271e-01,-5.59266448e-01],[-4.04839486e-01,2.19038222e-02,-9.21842337e-01],[6.15047216e-01,1.66971283e-03,-1.17130928e-01]],[[1.66894212e-01,-4.36287597e-02,4.44693297e-01],[-1.25514260e-02,1.15629053e+00,5.07267535e-01],[-1.31998003e+00,-2.01068565e-01,4.43417847e-01]],[[-4.29275602e-01,-1.80342898e-01,-7.68625677e-01],[-4.44070011e-01,7.40519539e-02,8.55759904e-02],[-1.84697270e+00,-3.88931781e-02,-2.20357805e-01]],[[6.04799867e-01,-3.28000456e-01,-2.43817404e-01],[-4.45692509e-01,-3.98754299e-01,3.13158095e-01],[1.59139093e-02,9.39212561e-01,-6.01141751e-01]],[[4.66475815e-01,2.31039569e-01,-5.22569239e-01],[-4.51610178e-01,-1.52821571e-01,-1.61465019e-01],[-5.51829457e-01,-4.85561520e-01,2.31702477e-01]],[[3.81792784e-01,-1.50816143e-01,5.85663319e-02],[-2.05522895e-01,2.99888492e-01,-5.93801320e-01],[1.11101055e+00,-1.60469085e-01,-3.17247182e-01]],[[-3.41899283e-02,4.94104207e-01,1.46976873e-01],[3.57184052e-01,-3.18863094e-01,5.23074210e-01],[-6.80166543e-01,-3.86027813e-01,-2.51428902e-01]],[[-2.94426531e-02,-2.71479815e-01,1.64280355e-01],[-2.06873372e-01,-1.21831425e-01,1.62591320e-02],[8.27933609e-01,-8.12987328e-01,4.95928168e-01]],[[-2.88402557e-01,2.09241748e-01,2.25368604e-01],[-4.10450622e-02,4.19690281e-01,3.79795611e-01],[6.05497599e-01,-4.88810018e-02,3.62624198e-01]],[[-1.70673713e-01,-4.16901439e-01,-8.77764583e-01],[2.06427678e-01,3.78876850e-02,1.92787364e-01],[4.21185732e-01,2.32644185e-01,-5.98163366e-01]],[[1.86981285e+00,1.51741222e-01,-1.36861190e-01],[9.82871503e-02,-5.20808280e-01,8.44744593e-03],[2.37990767e-01,1.74458712e-01,4.01232392e-01]],[[5.09917200e-01,-1.54786864e-02,1.61996335e-01],[3.09424531e-02,-2.20217600e-01,-2.10987758e-02],[-5.48435263e-02,3.25333208e-01,-2.74592131e-01]],[[4.20532912e-01,-1.00809939e-01,-2.00432003e-01],[-1.55879885e-01,7.35997736e-01,-5.90189658e-02],[-4.02266026e-01,-2.14569837e-01,-2.86905348e-01]],[[6.88314259e-01,3.07879597e-01,5.70444047e-01],[3.09337944e-01,-1.19309850e-01,3.31959128e-01],[7.46332228e-01,1.47455588e-01,-3.11468065e-01]],[[-4.22167182e-01,-4.61593410e-03,-1.98413223e-01],[-5.52126586e-01,-6.00709736e-01,-3.09580117e-01],[3.35173070e-01,5.46346128e-01,-3.05523098e-01]],[[-8.03652480e-02,3.11048776e-01,1.43785283e-01],[-6.29126549e-01,2.87432313e-01,8.58350337e-01],[5.76062977e-01,7.35438824e-01,4.65868413e-01]],[[-2.54593790e-01,-5.74785233e-01,5.70739396e-02],[4.07350570e-01,-8.95521194e-02,-8.63107666e-02],[-4.04737800e-01,1.76901743e-01,6.48356795e-01]],[[-7.88648725e-01,-5.05104721e-01,3.14094186e-01],[-2.01181978e-01,-7.18930066e-01,1.31926268e-01],[4.61558878e-01,2.22512141e-01,-1.43531844e-01]],[[-8.72094929e-01,-1.53955007e+00,-1.13471232e-01],[-1.45736694e-01,1.56551406e-01,-1.07729292e+00],[6.07023947e-02,-6.81797862e-02,-1.25454748e+00]],[[2.18346298e-01,-2.52995402e-01,5.10961175e-01],[-2.27534607e-01,1.21352486e-01,-5.44918537e-01],[1.69758070e-02,7.94903815e-01,6.31385088e-01]],[[-1.19263344e-01,3.00640404e-01,8.00354034e-02],[1.34008139e-01,-5.92478037e-01,5.49735129e-01],[7.80555606e-01,7.70888388e-01,5.02158165e-01]],[[6.66811466e-02,-2.14534745e-01,-4.27672684e-01],[2.93216370e-02,6.54502988e-01,-2.90276974e-01],[3.69252741e-01,2.92877853e-01,-3.72137845e-01]],[[-1.15302011e-01,-2.31180117e-01,1.22560620e-01],[3.87921277e-03,-5.51092684e-01,-7.03177929e-01],[-1.16308659e-01,4.54399921e-02,5.40408254e-01]],[[-3.56085181e-01,7.64910802e-02,-4.58520681e-01],[-5.87601624e-02,-1.03346974e-01,-2.88458228e-01],[-4.69042301e-01,-2.90718913e-01,-6.14623010e-01]],[[5.50044440e-02,9.70717445e-02,-3.44514221e-01],[-1.08670086e-01,3.51254731e-01,-5.79943299e-01],[8.57786596e-01,-3.13861728e-01,-1.87335089e-01]],[[-6.27211779e-02,3.34832892e-02,4.30528931e-02],[-1.79737419e-01,-3.43670726e-01,-1.95179448e-01],[1.56081989e-02,-6.14962950e-02,1.96376443e-01]],[[7.28636235e-02,-5.58342934e-02,2.10769802e-01],[4.96468320e-02,-4.20468375e-02,-9.44520682e-02],[1.05133951e+00,1.20924056e-01,4.63059247e-01]],[[-1.57869324e-01,1.67207062e-01,-3.63590479e-01],[-4.04899657e-01,-7.85075277e-02,-3.76666039e-01],[-5.07053673e-01,5.79758525e-01,-1.19056329e-01]],[[4.35201854e-01,-2.25749314e-01,-3.76015812e-01],[-5.01879230e-02,-5.05364716e-01,-1.35664225e+00],[-2.79150695e-01,5.24567723e-01,8.52560857e-04]]],[[[4.52137440e-01,-9.08401191e-01,-5.59280217e-01],[-3.63538355e-01,-3.60716097e-02,-2.86703587e-01],[2.06856534e-01,-1.46451369e-01,-1.12268098e-01]],[[4.72544372e-01,4.58284080e-01,-3.92559141e-01],[-6.86650336e-01,1.01095605e+00,-2.45619535e-01],[9.26164165e-03,-4.51436758e-01,-3.02499086e-01]],[[-5.61767936e-01,-4.59299274e-02,3.14247131e-01],[-3.72306347e-01,7.85435259e-01,-6.81558549e-01],[3.02546829e-01,5.06488085e-01,1.03034377e+00]],[[4.38387096e-01,-1.66300103e-01,5.64380996e-02],[4.95434761e-01,-5.28572738e-01,1.06359914e-01],[-3.38640928e-01,4.73281115e-01,-8.89344573e-01]],[[-5.27776062e-01,-5.32339096e-01,4.53166395e-01],[-6.23239242e-02,3.24324191e-01,-1.67262971e-01],[-1.33269358e+00,2.82111704e-01,-1.57954648e-01]],[[3.61911893e-01,-6.65093422e-01,2.13964179e-01],[-5.19645751e-01,-1.38438091e-01,-9.38060522e-01],[1.38515187e-02,6.80197895e-01,7.41660655e-01]],[[-2.43881583e-01,-3.82397659e-02,5.29384837e-02],[-4.12694812e-01,-2.69819107e-02,3.18012655e-01],[-3.63447100e-01,1.17053080e+00,6.32613003e-02]],[[-2.50159174e-01,-2.60707140e-01,8.74595866e-02],[-3.93395752e-01,-3.50855365e-02,1.64027512e-01],[2.50881612e-01,3.04339916e-01,-8.36170912e-01]],[[-1.66215580e-02,-1.07083499e-01,-8.26775432e-01],[2.22723812e-01,3.78322929e-01,-5.88684201e-01],[5.56645453e-01,6.25162244e-01,7.18352199e-01]],[[-4.17252958e-01,2.23020371e-03,3.31602693e-01],[2.89804824e-02,-2.34760448e-01,-5.43087959e-01],[3.06662023e-01,-1.67397112e-01,6.92221284e-01]],[[-4.83501673e-01,3.06709856e-01,-6.31196618e-01],[6.81772977e-02,-3.23688120e-01,9.26087618e-01],[-3.93072516e-01,-1.97186828e-01,5.69894731e-01]],[[-2.34140396e-01,-5.14444292e-01,5.25410354e-01],[-6.22681156e-02,-8.50126892e-02,1.26880348e-01],[-7.35486984e-01,-8.08851898e-01,-3.62165093e-01]],[[-1.84308618e-01,-3.90353411e-01,-3.73792797e-01],[-5.82433581e-01,-3.18714708e-01,-8.38768423e-01],[1.33539751e-01,1.76101670e-01,-3.44555564e-02]],[[4.55685318e-01,-5.94172657e-01,5.46748117e-02],[-2.07936704e-01,6.28976747e-02,3.44013184e-01],[-3.53371292e-01,1.19209744e-01,-7.67854869e-01]],[[-8.51975456e-02,4.27581936e-01,1.26872331e-01],[-4.29856479e-01,-6.73720121e-01,-3.18111479e-01],[-5.12471437e-01,3.39079320e-01,-4.10566747e-01]],[[-1.26516744e-01,-1.84879974e-01,-1.78979114e-02],[2.09401757e-01,9.40611139e-02,1.44762278e-01],[-8.59663785e-02,7.00132906e-01,-1.78605601e-01]],[[-1.96059987e-01,-1.10861115e-01,-1.76760510e-01],[-1.34380805e+00,-1.11308686e-01,-3.16234648e-01],[4.28245008e-01,-3.03037763e-02,3.66918407e-02]],[[-1.85671404e-01,-2.21307933e-01,-1.35893479e-01],[-1.32796928e-01,2.12527856e-01,8.45757425e-02],[-4.28709462e-02,-3.59420218e-02,1.82722509e-01]],[[-6.96304813e-02,8.79728496e-02,-2.48612955e-01],[-9.86034796e-02,4.99909997e-01,-8.33878875e-01],[1.33554190e-01,2.76785105e-01,9.67648327e-02]],[[8.35483298e-02,3.84291857e-01,3.46130222e-01],[3.44574749e-01,-1.78252421e-02,-5.89797914e-01],[1.45238936e-01,-3.86430994e-02,-5.20248152e-02]],[[-5.52943945e-02,-3.06831390e-01,-4.29267213e-02],[-2.23785535e-01,-7.32732043e-02,-3.96957189e-01],[-5.32408476e-01,1.84715599e-01,-9.48712379e-02]],[[7.22952664e-01,5.51773846e-01,-1.55297086e-01],[5.30102476e-02,9.05182883e-02,1.55305967e-01],[-1.49807349e-01,1.86468408e-01,2.69314349e-01]],[[6.94022477e-01,-2.88674533e-01,1.41214237e-01],[-8.92735571e-02,-1.09741308e-01,-1.66617081e-01],[-2.32855916e-01,-5.43966889e-02,3.18322897e-01]],[[-4.93285894e-01,2.85689741e-01,7.19938159e-01],[-1.76369846e-01,-5.77216625e-01,4.84632045e-01],[-7.57580519e-01,6.41313016e-01,-3.74612212e-01]],[[-7.06396401e-02,-3.89358073e-01,3.27898115e-01],[8.67243186e-02,5.81541777e-01,-1.07173491e+00],[3.50939810e-01,-3.77870113e-01,-2.70757377e-01]],[[-9.49534118e-01,3.40456039e-01,-1.10883605e+00],[6.37100264e-02,-3.21375817e-01,1.63507640e-01],[-6.38690293e-02,3.78480256e-01,4.18358929e-02]],[[-1.67370513e-01,-1.56621188e-01,-6.60199583e-01],[1.74115986e-01,1.83548287e-01,-1.73958585e-01],[-3.59548002e-01,3.80755365e-01,1.43813297e-01]],[[2.45040894e-01,4.92113560e-01,-2.63306886e-01],[-4.89784807e-01,1.59079544e-02,1.93345830e-01],[2.42906952e+00,7.32817829e-01,2.04031849e+00]],[[-4.48612660e-01,4.14029025e-02,3.38778526e-01],[-2.93153852e-01,-1.15832485e-01,-7.74378300e-01],[-1.57301724e-01,6.23479187e-01,-3.33785892e-01]],[[-5.76096952e-01,1.35714278e-01,-2.09293626e-02],[-1.27599180e-01,-4.62267756e-01,6.04226768e-01],[3.99902850e-01,4.32919711e-01,1.40338466e-01]],[[2.43040696e-01,-2.60337949e-01,-2.55722344e-01],[5.71937919e-01,4.42923605e-01,3.27233702e-01],[-6.99121475e-01,1.71988085e-01,-4.69693154e-01]],[[-7.02119246e-02,6.43575341e-02,6.18065417e-01],[-6.30521774e-01,-5.58118165e-01,-4.62313592e-01],[5.62294781e-01,3.48465979e-01,-5.36600232e-01]],[[2.67169327e-01,-7.74663448e-01,-6.56965792e-01],[2.18337014e-01,6.26241922e-01,-9.20603096e-01],[-6.90513998e-02,1.42713457e-01,2.39734903e-01]],[[-1.53461859e-01,-8.69842410e-01,-3.94523680e-01],[6.70323014e-01,-5.05356610e-01,2.44559497e-01],[-2.49715149e-01,3.44851881e-01,-1.02954552e-01]],[[4.19112593e-01,-4.71186787e-02,2.38462277e-02],[2.84998029e-01,-3.81032050e-01,2.77538747e-01],[-1.55787602e-01,1.60569206e-01,1.96565911e-02]],[[-6.75868750e-01,-2.22023442e-01,-4.29635160e-02],[-2.34381437e-01,-2.83824682e-01,-4.46588218e-01],[4.98261191e-02,1.33733496e-01,-2.99937904e-01]],[[3.96076180e-02,-1.06983878e-01,-2.22450003e-01],[1.08029318e+00,7.72580266e-01,-2.50882596e-01],[2.60805398e-01,1.13439955e-01,-7.84269124e-02]],[[4.27862197e-01,-1.79003090e-01,3.08159083e-01],[-2.37133607e-01,1.43038899e-01,3.54816526e-01],[1.02981649e-01,1.41259700e-01,-1.47879884e-01]],[[-1.00402892e+00,2.69846112e-01,-2.19083428e-01],[2.53391832e-01,4.24893312e-02,-2.52091676e-01],[1.68564335e-01,3.27471197e-01,3.80667478e-01]],[[3.56346756e-01,1.58670723e-01,-2.21519977e-01],[2.54348010e-01,-3.01428586e-01,-8.20310295e-01],[-5.72544396e-01,7.89832920e-02,-3.71794879e-01]]],[[[9.60024059e-01,5.41079521e-01,1.22990780e-01],[-8.36373642e-02,-2.89639056e-01,-2.93027967e-01],[1.35439053e-01,-5.57170548e-02,1.15776390e-01]],[[1.45746246e-02,8.00678208e-02,4.23436752e-03],[8.84158071e-03,-4.17531967e-01,5.80556273e-01],[-5.70152402e-01,5.20206571e-01,-1.18924476e-01]],[[1.85418293e-01,-3.48030865e-01,-2.31768101e-01],[8.78438056e-02,-3.89571518e-01,-1.43216431e-01],[1.92406669e-01,1.54976919e-01,2.80581534e-01]],[[-8.61637890e-01,4.56210703e-01,-1.72753602e-01],[-4.48970981e-02,3.49500209e-01,4.25795078e-01],[-7.77194619e-01,1.89054132e-01,7.19413817e-01]],[[1.75841644e-01,-1.81859180e-01,4.09368128e-01],[-2.59024322e-01,3.95418666e-02,6.45337284e-01],[4.84932512e-01,3.40067327e-01,-5.67197278e-02]],[[1.83472306e-01,7.35361502e-02,2.90874004e-01],[1.41601726e-01,5.72970033e-01,-8.24337244e-01],[3.55098844e-01,-8.23973194e-02,-6.30793512e-01]],[[5.95117152e-01,7.33325630e-02,-5.10412872e-01],[8.42296183e-02,-7.10349023e-01,-1.89628825e-01],[3.10806632e-01,-5.29420562e-02,1.26799956e-01]],[[-5.87249935e-01,-2.67844707e-01,-8.23903009e-02],[-4.27878112e-01,1.22153006e-01,-2.54992366e-01],[2.93891132e-01,1.01381660e+00,3.30501616e-01]],[[4.68471088e-02,3.02404523e-01,1.98906600e-01],[-3.83969873e-01,4.54934776e-01,-6.58070266e-01],[3.19232285e-01,2.44168162e-01,-5.82726657e-01]],[[2.23760866e-02,5.30911326e-01,1.40296996e-01],[-1.21267629e+00,-8.63526016e-02,7.59591520e-01],[-3.98510605e-01,1.23646177e-01,-7.02641457e-02]],[[-2.40556419e-01,3.55960667e-01,2.20924124e-01],[6.48312941e-02,-1.58306938e-02,5.00616312e-01],[-2.57770956e-01,-5.55409551e-01,3.38083506e-01]],[[1.63201407e-01,-1.50745004e-01,5.40412784e-01],[-1.23472057e-01,1.37795866e-01,6.45265162e-01],[1.30158037e-01,1.27393723e-01,-2.96283096e-01]],[[4.78907168e-01,-2.07938060e-01,-3.70496511e-01],[6.10729992e-01,1.74904108e-01,1.63830131e-01],[8.83763552e-01,1.31040797e-01,-5.78875065e-01]],[[7.82733679e-01,-1.27852380e-01,1.34435510e-02],[3.49950552e-01,2.91985244e-01,2.25360706e-01],[6.60092056e-01,7.68581033e-02,-6.29321113e-02]],[[5.19314170e-01,8.68061244e-01,2.85227567e-01],[-3.70354414e-01,-1.04490146e-01,1.77362725e-01],[-2.63340980e-01,1.17187813e-01,-4.80120659e-01]],[[-8.16481948e-01,1.37019169e-03,1.71504512e-01],[-1.96142241e-01,9.68762040e-02,6.97138488e-01],[1.02312434e+00,-2.77728170e-01,4.08770621e-01]],[[-2.83490241e-01,-3.46766450e-02,2.50541538e-01],[-2.60667428e-02,-4.60987896e-01,-3.69522929e-01],[-6.60717666e-01,-6.11812025e-02,4.11455572e-01]],[[1.16036884e-01,4.27687496e-01,5.33560753e-01],[-7.09343031e-02,3.06469984e-02,-7.75289163e-02],[1.22931731e+00,1.45333573e-01,2.00559109e-01]],[[3.84954095e-01,-5.01763411e-02,-1.70320049e-01],[3.97930183e-02,-1.05468668e-02,-1.25492573e+00],[-5.22976339e-01,-2.90973425e-01,-9.65564430e-01]],[[-3.76164973e-01,-2.02916265e-01,1.55712411e-01],[2.39337757e-01,-8.73638988e-02,-6.44241571e-01],[-2.00408384e-01,-1.07865259e-02,-1.39714110e+00]],[[2.43620425e-01,-4.48302388e-01,6.47329092e-01],[-2.32765102e-03,-6.16701007e-01,7.65245020e-01],[-4.18065369e-01,4.74528611e-01,2.02384904e-01]],[[-1.05479467e+00,1.78584635e-01,-9.87959921e-01],[-2.73514539e-01,4.65618551e-01,-4.23011124e-01],[-6.14839867e-02,1.72447085e-01,-2.67558061e-02]],[[-9.17080939e-01,-4.77251291e-01,-2.25517321e+00],[3.74699906e-02,6.11957023e-03,-8.52700531e-01],[1.85561106e-01,-3.86453658e-01,4.76941792e-03]],[[-2.75139301e-03,1.97485387e-01,4.32076693e-01],[3.73461455e-01,9.35380682e-02,-4.09593105e-01],[-4.25563186e-01,-2.42957488e-01,-2.95272082e-01]],[[-4.88515168e-01,-2.41524309e-01,-5.33945799e-01],[-5.66240191e-01,7.65719563e-02,-5.08438408e-01],[-9.48612571e-01,1.79085910e-01,-5.79500675e-01]],[[-1.33936852e-01,1.50427461e-01,2.63665974e-01],[1.15725972e-01,-2.96900123e-01,4.01915759e-01],[4.57877696e-01,1.35971844e-01,-5.58825850e-01]],[[-6.32181615e-02,-1.91349968e-01,-5.54427624e-01],[9.87030342e-02,-7.43618980e-02,-5.37399471e-01],[-6.64544851e-02,4.55012992e-02,-1.13696015e+00]],[[-3.22394893e-02,1.70516238e-01,-8.56682509e-02],[2.44362742e-01,-3.41063052e-01,-6.57824218e-01],[-2.98577845e-01,2.46273503e-01,-9.56016481e-02]],[[1.77729055e-01,-9.29166079e-02,1.80514777e+00],[-3.98537181e-02,1.12044953e-01,8.25024769e-02],[4.26087499e-01,4.22946125e-01,4.89061952e-01]],[[2.21608341e-01,2.64030933e-01,1.91247404e-01],[4.40766364e-02,4.08872396e-01,-1.93336606e-02],[1.12107980e+00,-2.89726228e-01,9.84889567e-01]],[[8.98358405e-01,-3.04461777e-01,3.33967745e-01],[8.37336928e-02,-6.39800191e-01,-5.08052744e-02],[3.39678109e-01,6.24211058e-02,5.87934911e-01]],[[5.77060401e-01,2.94703245e-01,3.33569527e-01],[1.03334320e+00,2.77835399e-01,9.25026163e-02],[-2.43795589e-01,2.93106854e-01,-1.42656684e-01]],[[-2.82611132e-01,-1.98293328e-01,-1.12232134e-01],[-1.21423207e-01,-4.91458237e-01,-4.27111238e-01],[-2.77828634e-01,7.32533708e-02,1.32193968e-01]],[[-5.15655816e-01,-1.24203779e-01,-1.23743281e-01],[-7.97065254e-03,4.50117551e-02,3.10893953e-01],[3.74895334e-01,3.75250243e-02,9.16109681e-01]],[[9.71763283e-02,2.46476963e-01,9.89820156e-03],[7.52393067e-01,-7.76704252e-02,3.50304812e-01],[-1.84291348e-01,-1.57398492e-01,6.40092731e-01]],[[-3.58181745e-01,-3.30450833e-01,5.65342188e-01],[-4.54958081e-02,-1.50400788e-01,-4.41350311e-01],[-1.12538233e-01,-1.09683514e-01,-4.07003045e-01]],[[7.72330165e-01,-3.61710995e-01,5.00150323e-01],[-1.47874892e-01,-8.11523378e-01,-1.19313560e-01],[-6.84638321e-01,-5.64190686e-01,2.24536091e-01]],[[-1.63532838e-01,-7.10184455e-01,-1.37900412e-01],[-8.94078612e-01,9.27803144e-02,-1.68733925e-01],[-1.29938042e+00,4.41854626e-01,-2.42398486e-01]],[[3.84722948e-01,5.07056057e-01,-1.85065176e-02],[4.75996673e-01,3.98802072e-01,-3.86713833e-01],[-1.67301729e-01,1.39759295e-02,-4.88438495e-02]],[[-1.10563564e+00,-4.48165625e-01,-5.40259242e-01],[-1.44932479e-01,1.78518549e-01,-3.73251617e-01],[-7.18365431e-01,-2.11138278e-01,-1.48789167e-01]]],[[[-2.53584027e-01,-9.16497350e-01,-2.27550060e-01],[3.27314958e-02,1.99005798e-01,-2.34150216e-02],[-1.44251481e-01,6.90630376e-02,9.55183387e-01]],[[-4.78881299e-01,5.03338687e-02,-1.77462503e-01],[-6.21472895e-02,7.67116725e-01,-3.63169074e-01],[-1.72790289e-02,5.82233250e-01,-2.19019294e-01]],[[-9.30018067e-01,9.95472729e-01,7.49073982e-01],[-4.11513448e-01,4.80450571e-01,7.82960773e-01],[3.06971699e-01,-1.84892833e-01,4.94847834e-01]],[[4.94063586e-01,-6.80491686e-01,6.86044618e-02],[-3.36524785e-01,-3.51417601e-01,-2.38826901e-01],[3.45231108e-02,5.69852106e-02,-8.40683758e-01]],[[1.83877349e-01,2.17002034e-01,-6.29906237e-01],[2.37747326e-01,-4.13761348e-01,-7.18484214e-03],[4.27620620e-01,1.00558788e-01,-7.06182301e-01]],[[6.90259457e-01,3.02178413e-01,5.86142361e-01],[2.25580812e-01,-6.90875649e-01,-6.96604371e-01],[9.76778328e-01,-5.04905403e-01,-5.59488297e-01]],[[-3.33527654e-01,2.03401238e-01,-5.65574951e-02],[-5.00623062e-02,-3.34749311e-01,1.41781583e-01],[-2.50740707e-01,2.73868591e-01,-5.98441362e-01]],[[-2.05474436e-01,1.27212822e-01,2.83640832e-01],[1.53862819e-01,2.56095350e-01,-1.61685303e-01],[1.25788882e-01,-4.74910215e-02,6.70003176e-01]],[[-2.25365207e-01,-2.65877634e-01,-2.24578276e-01],[-4.88640964e-01,8.99589062e-01,-4.84192520e-02],[-2.74977267e-01,-2.53761858e-01,-2.81441718e-01]],[[4.45945673e-02,-4.82085615e-01,2.55276281e-02],[7.21066445e-02,8.33853409e-02,-5.66342652e-01],[3.73804957e-01,-3.77341136e-02,1.85550526e-02]],[[-3.20045173e-01,-1.53079048e-01,-3.48197192e-01],[1.30491927e-02,-2.80135185e-01,-1.84782773e-01],[-4.64283168e-01,-9.30310726e-01,3.75751813e-04]],[[1.11697145e-01,-3.53916705e-01,-2.62362622e-02],[-3.77571993e-02,-8.52647960e-01,4.94772971e-01],[2.40537897e-01,-3.32081676e-01,-1.80445120e-01]],[[-1.58425152e-01,2.55665421e-01,1.43537238e-01],[-4.31750208e-01,5.71306385e-02,-3.58176351e-01],[-2.02312127e-01,-1.86084390e-01,-1.47010708e+00]],[[2.23107591e-01,6.39165863e-02,2.94787399e-02],[-2.29615763e-01,1.28690913e-01,3.55145484e-02],[-1.12606382e+00,1.14541955e-01,3.87172140e-02]],[[-8.25332940e-01,-1.32727146e-01,-1.08171785e-02],[-5.59685528e-01,8.40173900e-01,2.48397905e-02],[-5.60803294e-01,-5.44389844e-01,-4.74771895e-02]],[[1.14353859e+00,2.51385331e-01,-1.44126028e-01],[-1.03176975e+00,-2.21961871e-01,-2.86759794e-01],[-6.09357879e-02,7.38717914e-02,3.27503592e-01]],[[-9.50002551e-01,-1.48480266e-01,2.04146683e-01],[6.35138154e-02,-2.16708571e-01,2.74697691e-01],[-4.82119948e-01,3.27662051e-01,3.12103629e-01]],[[-7.63375461e-01,-5.40217221e-01,-7.16233432e-01],[2.64155027e-02,-9.43000078e-01,-2.79779762e-01],[-2.28479914e-02,5.21195054e-01,-1.30092442e-01]],[[-3.04549217e-01,-3.91385764e-01,-9.03471839e-03],[-1.97543483e-02,1.27265483e-01,2.17038840e-01],[-1.55057907e-01,-4.37211096e-01,1.28322914e-02]],[[1.73924991e-03,-3.22602987e-01,-4.19592798e-01],[-4.17757899e-01,5.32935381e-01,-2.88819354e-02],[-2.42989838e-01,-2.00390145e-01,3.34172577e-01]],[[-1.28980368e-01,2.40456447e-01,3.90730679e-01],[-9.93863195e-02,1.23910354e-02,-4.65664506e-01],[3.73860449e-01,5.23686372e-02,1.52845278e-01]],[[7.88531840e-01,-4.50360626e-01,8.65430295e-01],[1.79765806e-01,1.97029769e-01,-1.40958831e-01],[-2.25683585e-01,-4.91537631e-01,2.28503361e-01]],[[-6.58199145e-03,3.05558369e-02,2.07731962e+00],[1.14590138e-01,1.10038511e-01,-3.83197427e-01],[-8.28026682e-02,2.65128314e-01,2.78643489e-01]],[[4.74031508e-01,1.56233847e-01,-8.00757825e-01],[-1.30069494e-01,1.46728218e-01,3.15398484e-01],[2.90582657e-01,3.78134340e-01,2.04232521e-02]],[[9.52172210e-04,2.44915895e-02,1.43440947e-01],[-9.89033803e-02,7.07829371e-03,5.16829919e-03],[-1.59528181e-01,-1.79220289e-01,6.86487317e-01]],[[1.89831048e-01,6.12036027e-02,5.57196498e-01],[-1.11730859e-01,-1.42648637e+00,-6.43623710e-01],[-6.24816835e-01,5.39775372e-01,-4.85531718e-01]],[[1.79144442e-01,2.03846574e-01,3.93438429e-01],[-2.80316710e-01,-6.93213660e-03,-8.06810975e-01],[3.57782573e-01,-8.71070400e-02,-8.76747370e-02]],[[6.07433021e-01,2.22067088e-01,7.29614139e-01],[-9.79799926e-01,3.45436633e-02,3.34024698e-01],[2.76601404e-01,-6.20476723e-01,5.55165589e-01]],[[-7.57808864e-01,1.04488291e-01,-1.30812192e+00],[-4.22310829e-02,-6.15181088e-01,-5.69620550e-01],[-1.61694676e-01,-3.39998186e-01,-8.32604349e-01]],[[-2.02693511e-02,-3.93339932e-01,5.35220802e-01],[-7.59838402e-01,1.46359622e-01,1.50622928e+00],[-7.31318891e-01,-6.96286142e-01,-2.24972412e-01]],[[-2.99217820e-01,9.54062164e-01,-2.48927489e-01],[-2.19247624e-01,3.27570051e-01,-7.98870847e-02],[1.49585545e-01,-2.74890698e-02,-1.68056756e-01]],[[-1.14854671e-01,-5.84924936e-01,4.68912095e-01],[-1.05495989e+00,8.26451331e-02,8.41666400e-01],[-4.33457792e-01,-7.82945514e-01,-4.15760458e-01]],[[1.20342627e-01,-1.78279623e-01,1.32428601e-01],[1.14918269e-01,2.29949594e-01,1.78878322e-01],[4.46031570e-01,6.13141835e-01,2.45557964e-01]],[[5.18341810e-02,8.46776068e-01,3.11935335e-01],[-4.52281237e-01,1.01240110e+00,2.78486609e-01],[7.70719647e-01,-4.52782512e-01,-6.87505528e-02]],[[2.02795610e-01,-3.31429392e-01,-3.33125263e-01],[-1.63091198e-01,-1.95925787e-01,-4.40987766e-01],[1.14684865e-01,-1.45332739e-01,-7.71826565e-01]],[[-9.68197435e-02,5.17062962e-01,-6.07576430e-01],[6.94529235e-01,-5.13985276e-01,4.42629635e-01],[-3.84500414e-01,-1.71631470e-01,-5.79270303e-01]],[[-3.10331136e-02,1.68283850e-01,-2.57320851e-01],[-1.83509797e-01,-5.43335266e-03,-2.85204411e-01],[-6.02734149e-01,-5.74105322e-01,3.68996173e-01]],[[4.34066534e-01,1.49272427e-01,1.99595064e-01],[1.51473179e-01,-1.57815754e-01,-1.88149646e-01],[5.58023930e-01,-7.00481459e-02,6.03445582e-02]],[[1.94199607e-01,2.01310396e-01,-9.75784719e-01],[3.50853473e-01,3.02493647e-02,5.03447473e-01],[-1.02640428e-01,-8.08685601e-01,-8.59830618e-01]],[[6.29716337e-01,2.99559683e-01,-2.99903214e-01],[4.42228943e-01,-2.16362342e-01,3.89233261e-01],[6.79107189e-01,-2.34404221e-01,-1.18580516e-02]]],[[[5.52268147e-01,4.00927216e-01,-1.00073658e-01],[3.17914277e-01,-2.15688750e-01,-1.46639213e-01],[2.28345841e-01,5.02206504e-01,-1.49741590e-01]],[[-5.43497145e-01,-1.99058056e-01,3.46876502e-01],[2.39115864e-01,-1.98456407e-01,3.30501288e-01],[-3.30503106e-01,1.66575387e-01,4.36871558e-01]],[[1.34456515e+00,-5.46452463e-01,-4.66552675e-01],[9.58590955e-02,-1.25280142e-01,-6.29225280e-04],[-4.93994564e-01,-4.11754176e-02,-9.92438272e-02]],[[3.59284371e-01,-5.45528710e-01,-2.17867166e-01],[-7.00728953e-01,4.06369455e-02,-6.83690786e-01],[-4.75122005e-01,6.80132091e-01,-8.27373445e-01]],[[-2.12604076e-01,6.55807629e-02,2.22734332e-01],[-7.86636829e-01,-4.12200727e-02,-1.28494665e-01],[-9.11841869e-01,5.38287982e-02,3.30240577e-01]],[[2.11009949e-01,3.48516852e-01,-2.29836419e-01],[2.91613638e-01,-2.21672565e-01,1.71067208e-01],[3.40627104e-01,1.86690837e-01,5.98210096e-01]],[[9.39134836e-01,6.55738264e-02,3.90208870e-01],[1.15355146e+00,2.39886746e-01,-8.98810029e-01],[6.90560341e-01,-9.71911252e-02,4.81934667e-01]],[[-3.06226969e-01,4.57327180e-02,-2.79466938e-02],[-1.31338552e-01,-2.54614949e-01,-1.15930706e-01],[6.52790844e-01,-1.15102418e-01,-3.45621407e-01]],[[8.36149510e-03,-3.02857816e-01,-9.40200031e-01],[3.07453573e-01,-3.38630170e-01,-4.18966502e-01],[-3.12656254e-01,3.07006836e-01,-3.52278590e-01]],[[-3.89988780e-01,1.45928398e-01,7.76501358e-01],[-1.83010265e-01,-3.35198522e-01,-3.26481313e-01],[-3.93743336e-01,3.93087119e-01,4.47251089e-02]],[[-1.85701087e-01,-3.98417383e-01,3.77761275e-01],[-1.97379708e-01,2.29272172e-01,4.27896887e-01],[-6.77524686e-01,-1.66194454e-01,2.38140911e-01]],[[3.84164751e-01,-1.81030422e-01,-9.19295609e-01],[2.29023233e-01,-1.28563553e-01,2.06391886e-01],[7.00894818e-02,-1.02473783e+00,3.33303362e-01]],[[6.73302174e-01,-4.12261516e-01,1.51482701e-01],[-3.30312937e-01,-5.54100275e-01,1.30614549e-01],[-6.79944679e-02,4.35085371e-02,4.23911035e-01]],[[-4.88991410e-01,1.09504178e-01,5.55554926e-01],[-1.18235014e-01,-4.02547181e-01,-1.79945782e-01],[7.88930282e-02,-5.37837505e-01,3.02680016e-01]],[[5.37893772e-01,-7.93904997e-04,4.26049024e-01],[3.89775544e-01,-6.73144730e-03,6.83144927e-01],[4.79849279e-01,8.67345035e-02,2.33035818e-01]],[[1.82229295e-01,6.06143363e-02,-4.81834076e-02],[3.11179943e-02,4.80461530e-02,-5.37078619e-01],[7.34031320e-01,1.56234242e-02,-2.71541148e-01]],[[4.77744728e-01,-9.06264544e-01,8.18610564e-02],[3.45153213e-01,1.71916023e-01,2.79279560e-01],[2.12134153e-01,-2.02540219e-01,4.93089892e-02]],[[-1.13079751e+00,-4.94167842e-02,-2.00604513e-01],[-8.17643940e-01,-3.22918653e-01,1.62462428e-01],[-5.27360499e-01,-7.32481897e-01,5.65296888e-01]],[[4.52393085e-01,5.90449035e-01,-1.04288578e-01],[-5.12385428e-01,4.05856580e-01,4.85047430e-01],[-2.38977864e-01,8.17289501e-02,5.27049452e-02]],[[2.03700900e-01,-2.00221673e-01,-3.16307731e-02],[2.15837568e-01,-4.17676449e-01,-2.05482375e-02],[-2.90789306e-01,-3.44022572e-01,2.31265321e-01]],[[9.70170498e-01,1.27600566e-01,9.45279837e-01],[-2.55953986e-02,8.86948034e-02,5.18757820e-01],[-8.56516302e-01,-8.50158691e-01,5.36081731e-01]],[[5.89671671e-01,1.29727352e+00,-6.29949093e-01],[4.38796222e-01,-1.86344497e-02,4.50877666e-01],[8.61389190e-02,-5.35715580e-01,-3.73089761e-01]],[[1.88059673e-01,1.13352248e-02,1.93988174e-01],[-3.96896392e-01,4.12941486e-01,-2.00449333e-01],[6.37333095e-02,-5.74679017e-01,1.53447047e-01]],[[7.23174661e-02,-3.03403378e-01,-4.84267503e-01],[-1.15577802e-01,1.49810851e-01,-1.33487005e-02],[-9.96945679e-01,3.82694691e-01,3.83442342e-02]],[[-9.08425927e-01,-5.74163616e-01,-2.34545961e-01],[1.15834907e-01,2.27989197e-01,-3.90586108e-01],[-5.08395918e-02,-3.04871798e-01,-1.81132346e-01]],[[1.71598583e-01,6.72763050e-01,-1.19102865e-01],[2.77936637e-01,-1.38815567e-01,-2.52268642e-01],[-4.27548766e-01,4.48756991e-03,-4.48064655e-01]],[[6.69280469e-01,1.95422679e-01,3.03076953e-01],[7.65864621e-04,-2.43346319e-02,1.81005850e-01],[6.52932048e-01,3.70007932e-01,7.04417109e-01]],[[-9.60827395e-02,4.76293147e-01,-4.89535570e-01],[2.44788453e-01,-1.21845722e-01,-1.96887985e-01],[-5.12801290e-01,9.16130304e-01,-6.92321956e-01]],[[-8.05372596e-01,2.00420529e-01,-4.44047987e-01],[-3.42061520e-01,-1.91537619e-01,-1.85049884e-02],[2.44744629e-01,-3.50474983e-01,-4.22700457e-02]],[[6.83681667e-01,-2.12311342e-01,2.73803975e-02],[4.11947936e-01,-1.63043350e-01,7.06331968e-01],[-5.38169332e-02,3.64969730e-01,1.82661340e-01]],[[1.25786686e+00,-8.50014865e-01,2.21932149e+00],[2.07713470e-01,-7.45909154e-01,2.13911355e-01],[1.03620946e-01,-5.56504607e-01,6.86038375e-01]],[[9.45707738e-01,4.24432121e-02,-1.79688941e-05],[-2.77462393e-01,-8.83977592e-01,-1.35087681e+00],[5.87771297e-01,-1.94635242e-02,-4.67183203e-01]],[[1.03773427e+00,-2.47970507e-01,1.45110071e+00],[2.39795163e-01,4.20925617e-01,8.25464502e-02],[4.82768357e-01,1.54734537e-01,4.60686862e-01]],[[5.15632808e-01,8.49972516e-02,-1.35176018e-01],[2.29353681e-01,4.79339361e-01,1.92578077e-01],[-1.96241103e-02,1.53957130e-02,2.15700284e-01]],[[-2.19191596e-01,4.02377732e-02,-6.51263893e-01],[1.80796787e-01,3.90824795e-01,3.12004507e-01],[6.11918010e-02,3.05238277e-01,2.06790715e-02]],[[-1.42453030e-01,-5.72014213e-01,5.65458894e-01],[-1.21179193e-01,-4.63061213e-01,5.94954669e-01],[-2.23299675e-02,-4.95167494e-01,7.16417134e-01]],[[-4.12699401e-01,-5.32084763e-01,2.89517313e-01],[-4.25148785e-01,-1.79568045e-02,1.24340773e-01],[-7.89078593e-01,3.92416447e-01,1.28596663e-01]],[[-6.51670575e-01,1.12350118e+00,-3.12870070e-02],[-9.37147021e-01,-6.36222363e-01,-5.50713599e-01],[-4.61479694e-01,1.76505223e-02,1.12755217e-01]],[[-9.52683985e-02,5.33250809e-01,7.09938183e-02],[-6.47001922e-01,-8.17273915e-01,-1.16412915e-01],[-6.14035487e-01,-2.52553403e-01,-6.60231650e-01]],[[1.95336878e-01,2.99775451e-01,3.49109888e-01],[-5.53320587e-01,-4.04988438e-01,4.93246406e-01],[5.74336946e-02,-8.49341631e-01,-1.74665987e-01]]],[[[1.57482117e-01,1.28111154e-01,-8.55845809e-02],[4.20034826e-01,-4.98678327e-01,2.75409967e-01],[2.03450881e-02,-3.23738009e-01,7.23114386e-02]],[[2.02734262e-01,-4.59860489e-02,-1.91364646e-01],[1.39344275e-01,3.71505648e-01,-1.32690683e-01],[4.41369653e-01,4.14852291e-01,9.77146104e-02]],[[-5.50468981e-01,-2.94460133e-02,-4.15629111e-02],[1.52829304e-01,2.85079420e-01,3.01574647e-01],[-3.23050916e-01,-6.01325750e-01,3.04720670e-01]],[[-6.14664376e-01,2.31449157e-01,6.51956022e-01],[-2.23245934e-01,-2.38915160e-01,-1.25584360e-02],[-1.11145116e-01,5.08625686e-01,-3.06212716e-02]],[[-5.16955614e-01,3.75295401e-01,-4.15773124e-01],[3.73778790e-01,-8.13320160e-01,1.75575331e-01],[-6.73156202e-01,1.13503313e+00,5.44776380e-01]],[[-2.66745865e-01,-1.95820361e-01,1.79746121e-01],[7.78865159e-01,-5.15505254e-01,-1.86757430e-01],[9.81088758e-01,4.57351118e-01,-1.39995348e+00]],[[1.17082726e-02,-4.59539950e-01,-3.19223493e-01],[-8.35443556e-01,-3.09505910e-01,-3.36448789e-01],[3.86866659e-01,3.83429192e-02,-1.89688087e-01]],[[5.60334139e-02,3.88993591e-01,1.39708132e-01],[4.17647660e-01,-5.29292710e-02,2.98151731e-01],[-8.17420125e-01,3.75680506e-01,-2.17527837e-01]],[[3.20279509e-01,-1.90053597e-01,-2.76375949e-01],[2.50265747e-01,-4.82418090e-02,-6.87812805e-01],[3.35692167e-01,-7.81024575e-01,6.73184991e-02]],[[2.37974703e-01,-9.25172865e-01,2.02835184e-02],[5.05677044e-01,-2.97887981e-01,-2.43680984e-01],[1.25443265e-01,-3.90493453e-01,-3.06512386e-01]],[[-6.74206972e-01,-2.00024500e-01,-3.34485590e-01],[2.01882824e-01,1.29981786e-01,-1.34903444e-02],[-1.17949378e+00,-7.90755868e-01,-5.32515645e-01]],[[-3.16694647e-01,-4.85054523e-01,2.49067828e-01],[7.57875070e-02,3.50456610e-02,-2.06749842e-01],[1.37940884e-01,2.15018347e-01,2.25973818e-02]],[[4.13153529e-01,-5.34559190e-01,-1.27673998e-01],[2.12916300e-01,5.96415937e-01,7.21206784e-01],[3.98867339e-01,-1.63989753e-01,2.24916130e-01]],[[1.45914881e-02,-1.65118828e-01,5.07038273e-02],[-2.41606191e-01,5.13866603e-01,-4.78781670e-01],[3.83729100e-01,3.92202027e-02,-1.56086862e-01]],[[8.78390431e-01,2.96006531e-01,-4.06302989e-01],[2.75332391e-01,-5.12637012e-03,3.17190737e-01],[6.52895033e-01,-3.53524461e-02,2.06485868e-01]],[[-2.51510620e-01,-4.47446287e-01,2.18066886e-01],[-1.36160180e-02,-2.26218849e-01,-7.69727588e-01],[5.44975460e-01,-3.12856704e-01,-7.78049290e-01]],[[-2.10159302e-01,-1.94669701e-02,1.79191101e-02],[1.18995702e+00,1.69750318e-01,6.95448555e-03],[4.32533622e-01,-1.06274927e+00,4.60376263e-01]],[[-6.78402305e-01,-3.02750260e-01,-2.00271577e-01],[1.05662274e+00,3.28578055e-01,7.08337426e-01],[-1.33143321e-01,2.36503273e-01,5.36361635e-01]],[[-3.59367728e-01,-6.55952021e-02,-2.02928722e-01],[6.03129268e-01,1.08522248e+00,-2.29943499e-01],[5.79314530e-01,7.40489483e-01,-1.50863254e+00]],[[-8.47567260e-01,-3.42383862e-01,4.20781106e-01],[-2.08271444e-01,5.36520123e-01,-3.05321723e-01],[-7.48472631e-01,3.78000855e-01,-7.55009174e-01]],[[2.13692605e-01,1.72041953e-01,3.42748225e-01],[-3.18736613e-01,3.10307145e-01,-1.61427245e-01],[1.13909936e+00,1.55330688e-01,-1.07286346e+00]],[[-2.82235414e-01,1.05760479e+00,3.58758897e-01],[2.00009495e-01,1.00704372e-01,-1.35925949e-01],[-5.74935138e-01,-4.29678231e-01,7.44212568e-02]],[[-1.36288971e-01,-6.06570005e-01,6.47862395e-03],[2.47243911e-01,-9.00924131e-02,6.99592948e-01],[-4.69350159e-01,-3.59700143e-01,1.53345346e-01]],[[-1.69358924e-01,2.37342581e-01,9.83052179e-02],[2.64480591e-01,-1.87690496e-01,2.25311816e-01],[1.78906620e-01,5.70609331e-01,-3.53351444e-01]],[[-2.13403478e-01,3.36468726e-01,4.57646668e-01],[1.04444370e-01,3.10506150e-02,-5.60809374e-02],[-3.92432094e-01,-6.46609366e-02,1.92756861e-01]],[[-1.24883344e-02,9.02671833e-03,1.59064326e-02],[-4.43238944e-01,-3.30226421e-01,3.22935246e-02],[-1.13141108e+00,-1.62946075e-01,3.52999985e-01]],[[1.33448234e-02,-4.32978183e-01,3.16723317e-01],[-3.14820498e-01,2.10883602e-01,-5.03548086e-01],[2.37848550e-01,2.75799304e-01,-4.54561204e-01]],[[5.73112518e-02,3.73068415e-02,9.22186136e-01],[4.14361916e-02,4.05906230e-01,-8.00238013e-01],[5.28290905e-02,2.53589809e-01,7.76773810e-01]],[[3.83695990e-01,-5.39987504e-01,-1.73673317e-01],[3.68134230e-01,-2.04076365e-01,9.01168585e-01],[5.70048511e-01,-3.56687754e-02,6.37339056e-01]],[[1.13539904e-01,3.07288826e-01,-4.52841997e-01],[6.48514450e-01,-4.72730696e-01,3.08562577e-01],[-4.10345674e-01,-2.00950876e-01,-1.12569615e-01]],[[4.05828595e-01,7.14538619e-05,-2.29971677e-01],[3.12651068e-01,8.63226831e-01,3.23374748e-01],[2.66234607e-01,-2.75182307e-01,8.40642393e-01]],[[1.97326839e-01,-1.24323264e-01,-8.39285925e-02],[-4.64142472e-01,-1.09477603e+00,-1.99213192e-01],[-3.02950352e-01,-5.21229744e-01,3.00128490e-01]],[[-4.27032381e-01,-2.61332273e-01,-6.76764250e-01],[5.04605651e-01,-4.73614275e-01,1.92415357e-01],[3.92881602e-01,4.38108109e-02,-1.17962606e-01]],[[-1.20894456e+00,4.65180963e-01,-9.48251009e-01],[-4.34437878e-02,-8.46899450e-02,2.64573067e-01],[5.53282857e-01,6.20930910e-01,2.33150527e-01]],[[-1.57712772e-01,3.05840727e-02,-1.30186602e-01],[-1.04985675e-02,-2.63188779e-01,-5.95092714e-01],[9.75448936e-02,1.94424182e-01,-8.55953693e-02]],[[-3.99753213e-01,-5.56344688e-02,2.08675265e-01],[-4.87263352e-01,2.50242293e-01,-5.78119159e-01],[-4.68131870e-01,3.23768198e-01,-8.20132613e-01]],[[3.18320990e-01,-6.93761110e-02,-1.89472213e-01],[-4.35247645e-02,-9.81525108e-02,1.21867724e-01],[4.85287488e-01,-5.14652617e-02,-3.94541413e-01]],[[-7.96277404e-01,-3.98367375e-01,4.06763941e-01],[-4.71809179e-01,-4.75110531e-01,7.98211753e-01],[4.81470913e-01,-6.99896336e-01,-4.87706035e-01]],[[-1.08806717e+00,-1.66585743e-01,-7.62056589e-01],[1.59912005e-01,1.42947316e-01,-5.80796123e-01],[-6.22878850e-01,3.03560287e-01,5.33334613e-01]],[[-3.16052377e-01,-1.51397958e-01,1.82120368e-01],[8.03040117e-02,-6.51630342e-01,1.82080209e-01],[4.45980020e-02,-1.40192032e-01,-1.04968265e-01]]],[[[1.42165291e+00,-1.67157054e-01,4.36226368e-01],[3.17956209e-01,2.96273232e-01,2.43735146e-02],[2.08713770e-01,-5.63793540e-01,1.86690241e-01]],[[-1.51372120e-01,2.56487787e-01,4.30879176e-01],[2.41300389e-01,-3.84731948e-01,4.89468008e-01],[-2.60606050e-01,-2.57240176e-01,-5.59479475e-01]],[[-3.86766583e-01,3.63628238e-01,1.91945836e-01],[-1.95321620e-01,-1.25230923e-01,-6.90392971e-01],[-2.71950752e-01,-4.55809414e-01,4.35306162e-01]],[[-5.20808399e-01,-9.99822542e-02,-8.60282898e-01],[2.30732366e-01,-6.27655447e-01,1.71840653e-01],[-1.20146692e-01,-6.52684391e-01,3.08297008e-01]],[[-2.45450765e-01,1.11844100e-01,-1.13226950e-01],[1.41375452e-01,7.73065865e-01,-2.79598057e-01],[2.35579997e-01,9.22832608e-01,1.87753096e-01]],[[2.37433583e-01,-1.90892592e-02,6.37291312e-01],[3.20353150e-01,-5.41290224e-01,-5.23475826e-01],[6.12370133e-01,5.03116734e-02,-6.98907912e-01]],[[1.62341684e-01,1.73158020e-01,4.02461886e-01],[-5.19850254e-01,1.64421856e-01,2.77833417e-02],[6.27362669e-01,4.35382515e-01,2.60771066e-01]],[[-7.02411115e-01,1.31486908e-01,-1.41818061e-01],[1.02355731e+00,2.21069511e-02,-1.03032596e-01],[6.48207843e-01,-9.67721224e-01,3.83307904e-01]],[[6.71543088e-03,-2.45165169e-01,6.89582452e-02],[-5.23225009e-01,7.92778552e-01,5.71428239e-01],[-4.58458960e-01,-5.44949546e-02,2.51029670e-01]],[[5.29603183e-01,-7.66675398e-02,2.97255576e-01],[-5.56852996e-01,8.35749209e-02,3.04455340e-01],[-1.11838031e+00,6.36673942e-02,2.85893798e-01]],[[5.92029810e-01,-6.09862328e-01,-6.81471050e-01],[5.35574496e-01,2.55945381e-02,-5.48224151e-01],[-2.51551300e-01,8.83274898e-02,-6.08217537e-01]],[[1.24452591e-01,4.75645512e-02,4.66066897e-01],[1.67341471e-01,-2.32437655e-01,7.55211890e-01],[2.03103051e-01,2.14600772e-01,-2.73450464e-01]],[[-2.97407731e-02,-1.84202924e-01,-4.49796766e-01],[-1.75837368e-01,4.79300946e-01,-1.91218406e-01],[-4.90271866e-01,-2.85154104e-01,-6.22876048e-01]],[[-6.20466322e-02,7.76824951e-01,-6.55924022e-01],[8.03368390e-01,7.20606074e-02,2.17934236e-01],[7.55277812e-01,2.08362013e-01,3.55817556e-01]],[[8.27401340e-01,3.96750391e-01,3.01404238e-01],[-1.22530445e-01,-1.78402454e-01,2.80836105e-01],[-8.55860949e-01,3.77332509e-01,1.33453727e+00]],[[7.39273429e-02,-6.80339187e-02,4.25885022e-02],[-1.41137755e-02,3.08628649e-01,-2.71044284e-01],[-2.16132123e-02,-1.84403524e-01,7.94820666e-01]],[[-3.09174150e-01,2.06217960e-01,-2.95657009e-01],[-1.14046797e-01,-3.77502054e-01,-3.91565830e-01],[-7.36766040e-01,-1.68871418e-01,-5.74114203e-01]],[[5.06094158e-01,-2.77582586e-01,-2.18871221e-01],[-2.94047892e-02,-7.71954119e-01,3.87945265e-01],[-7.88936734e-01,-2.06768781e-01,1.51245189e+00]],[[-2.44539529e-01,3.50844949e-01,1.65797710e-01],[-8.10800344e-02,-6.88105449e-02,4.17241268e-03],[8.65899086e-01,-1.19645037e-01,-7.15070367e-01]],[[2.40917087e-01,-3.07154328e-01,-2.73837715e-01],[-1.59147918e-01,2.98306853e-01,-1.52714074e-01],[-1.28499806e+00,-6.45030320e-01,-7.84925401e-01]],[[2.76815116e-01,3.56104635e-02,2.82076269e-01],[1.31289989e-01,2.70329509e-02,4.53616306e-02],[1.04005992e-01,1.49747673e-02,-4.17960495e-01]],[[-6.99521244e-01,1.34304345e-01,1.57244503e-01],[-4.16791528e-01,-5.83047681e-02,-1.22149035e-01],[4.67452556e-02,-4.84478250e-02,1.72305003e-01]],[[-7.02212572e-01,1.18131749e-01,-7.87560403e-01],[7.90531099e-01,4.89344820e-03,4.88111496e-01],[-3.87811899e-01,6.16542101e-01,-1.38836294e-01]],[[2.93597251e-01,-7.00376481e-02,-4.30591047e-01],[6.07642233e-02,-1.14116939e-02,1.51085094e-01],[2.96404362e-01,-3.89599144e-01,-4.04137760e-01]],[[-2.81094939e-01,-5.89927197e-01,-4.08779025e-01],[-1.85582995e-01,-2.32135117e-01,6.85413659e-01],[-2.90426761e-01,-8.28820989e-02,-1.55485630e-01]],[[6.15489602e-01,1.95643678e-01,1.11801535e-01],[-1.97552487e-01,5.36558211e-01,-4.52115357e-01],[-2.58313894e-01,-7.85955250e-01,1.82561070e-01]],[[2.58027524e-01,1.59583971e-01,-5.48110843e-01],[1.74327269e-01,-3.37722264e-02,-2.89201617e-01],[2.25210756e-01,-5.97798713e-02,-7.30080187e-01]],[[6.53233349e-01,-1.41634926e-01,-4.77648884e-01],[7.18122646e-02,7.48572409e-01,7.61552691e-01],[-3.84210557e-01,1.75565347e-01,2.39537135e-02]],[[4.81150419e-01,5.83006620e-01,6.70151472e-01],[-2.58743484e-02,1.88122004e-01,-5.55867136e-01],[-4.62261558e-01,4.96530607e-02,-1.08300996e+00]],[[-1.53721392e-01,3.47624451e-01,-3.29561055e-01],[4.53068256e-01,-7.48347640e-02,-4.96119112e-01],[6.22117996e-01,-2.62424350e-01,3.82158637e-01]],[[1.55237734e+00,3.71376306e-01,-3.84283394e-01],[-1.75714716e-01,2.82857776e-01,-3.20695043e-01],[7.42667675e-01,3.01995605e-01,-6.26124516e-02]],[[-4.82935369e-01,6.53411865e-01,-1.16903074e-01],[-4.79052782e-01,1.51372463e-01,-8.13919902e-02],[-9.25517678e-01,-9.47138131e-01,2.31243655e-01]],[[-5.34135878e-01,1.34215038e-02,-4.72814947e-01],[-4.51241106e-01,3.12254339e-01,1.86132565e-01],[-2.62107342e-01,5.05211949e-01,-5.92095792e-01]],[[3.55470181e-01,-5.95356166e-01,-4.56543416e-02],[4.46114302e-01,4.82457340e-01,-2.05143154e-01],[7.48771846e-01,5.07081091e-01,7.41003573e-01]],[[2.32096672e-01,7.72299945e-01,7.21921086e-01],[1.31919846e-01,1.19890764e-01,3.71660769e-01],[-9.21223834e-02,7.62253881e-01,-5.88301197e-02]],[[-1.04349160e+00,1.86538860e-01,2.44264796e-01],[1.15066452e-03,5.08278131e-01,8.69272649e-01],[-4.39913124e-01,-1.77692235e-01,-7.21671879e-01]],[[-2.80068487e-01,-4.73153234e-01,7.64825106e-01],[-4.68226582e-01,-5.44076800e-01,3.43734056e-01],[1.10984348e-01,-2.35796526e-01,-4.45001125e-01]],[[-3.69850993e-01,4.05715741e-02,-4.04046535e-01],[-1.20499894e-01,1.64547279e-01,-6.14857562e-02],[-2.58787155e-01,3.80710155e-01,-1.31101266e-01]],[[6.10511601e-01,-3.63711804e-01,-4.21197206e-01],[3.69071126e-01,-2.28471652e-01,2.36707598e-01],[8.82239997e-01,-5.08713908e-03,7.60878902e-03]],[[-7.08152652e-01,-9.33443662e-03,1.85670644e-01],[2.45073929e-01,1.10960402e-01,-4.60529774e-01],[6.34257570e-02,-4.05301861e-02,-6.73629820e-01]]],[[[-5.41803598e-01,-3.87879997e-01,-7.74169207e-01],[-8.83935615e-02,-2.25965455e-01,2.65429974e-01],[-1.57429412e-01,-2.68026069e-02,-2.05404893e-01]],[[-1.68944746e-01,-5.37506461e-01,1.61988251e-02],[3.74738991e-01,-5.29020727e-01,3.05088967e-01],[9.14373100e-01,1.84254020e-01,-3.44965130e-01]],[[2.48630062e-01,8.39467168e-01,2.93651342e+00],[1.12772256e-01,2.79755533e-01,-4.76329997e-02],[-3.48990291e-01,3.77555370e-01,9.03884828e-01]],[[-1.94614097e-01,3.11499238e-01,-1.96041748e-01],[6.88498378e-01,-3.87890488e-02,-9.13910791e-02],[-1.20101981e-01,-2.47100487e-01,-2.21993387e-01]],[[-3.03516421e-03,-3.98991644e-01,-1.07054226e-01],[2.93049455e-01,1.18003361e-01,1.70809001e-01],[6.10406578e-01,1.99997842e-01,3.12584907e-01]],[[1.02957729e-02,-4.76374179e-01,2.78647512e-01],[-1.84212357e-01,-8.05908144e-01,-2.43588254e-01],[-5.44216871e-01,-7.56365582e-02,-3.35138857e-01]],[[2.63197333e-01,5.70497513e-01,-6.07189834e-01],[8.39998350e-02,1.45823896e-01,-8.07953030e-02],[-5.18465564e-02,-3.84701490e-01,4.76657599e-01]],[[1.89517125e-01,-4.68585104e-01,-1.59192428e-01],[-1.52488589e-01,3.04553479e-01,-1.19101971e-01],[-1.01973936e-02,-7.17895567e-01,-5.39292455e-01]],[[1.12627834e-01,2.52605408e-01,-3.67549539e-01],[4.09756508e-03,6.84294328e-02,-2.70041436e-01],[-4.61644113e-01,-8.61412942e-01,-3.22367817e-01]],[[4.19990093e-01,3.79928321e-01,3.63570511e-01],[6.30979598e-01,-1.43501058e-01,-4.95940864e-01],[1.69031625e-03,-4.55655277e-01,1.12374388e-01]],[[-5.34352176e-02,-3.27992499e-01,-1.75843567e-01],[-2.85910964e-01,-1.13857694e-01,1.01085931e-01],[1.99871182e-01,-6.23876691e-01,-4.95305687e-01]],[[2.38294810e-01,8.73208120e-02,9.00624514e-01],[-2.91501544e-03,-6.02674223e-02,6.06478989e-01],[-1.76869899e-01,-8.81182611e-01,-7.72338882e-02]],[[-7.94920549e-02,8.55135024e-02,-3.62180173e-01],[-1.65862173e-01,1.70541421e-01,-2.29965612e-01],[1.20341986e-01,-3.28389227e-01,-3.26850295e-01]],[[-3.72902960e-01,4.65070367e-01,8.93360138e-01],[2.67334104e-01,-4.07971023e-03,-2.14221671e-01],[-1.06195115e-01,-1.88198145e-02,-1.52993351e-01]],[[-7.83133209e-01,-6.45102188e-03,1.69792265e-01],[1.76123649e-01,-7.53475070e-01,-6.66409969e-01],[7.63869807e-02,-1.47399500e-01,-1.73050869e+00]],[[4.52781171e-01,-3.75036269e-01,-8.53499770e-01],[1.00604549e-01,9.91727412e-02,-6.11542225e-01],[3.04264873e-01,2.42058098e-01,-1.06187773e+00]],[[4.05750901e-01,8.25781301e-02,2.72986621e-01],[3.41130644e-01,3.27373147e-01,1.55136615e-01],[-1.45110816e-01,-8.41343477e-02,9.35450643e-02]],[[2.28295997e-01,-3.24443728e-01,-1.20287709e-01],[6.89399779e-01,-4.05748248e-01,7.57413357e-02],[2.34552905e-01,4.25374247e-02,9.02055129e-02]],[[6.52953535e-02,-5.60855925e-01,1.99420676e-01],[-3.22133571e-01,-1.17915165e+00,9.54496741e-01],[6.28288805e-01,9.48859036e-01,-3.41369390e-01]],[[7.55168498e-01,-2.40874901e-01,-5.67421854e-01],[4.56356287e-01,2.74677798e-02,-7.81539857e-01],[-1.16977572e-01,2.01608632e-02,-4.79753345e-01]],[[-3.49316448e-01,3.06422776e-03,1.58436537e+00],[-2.54030883e-01,1.38215423e-01,2.10600913e-01],[1.16679043e-01,8.71827602e-02,1.13055050e+00]],[[-1.01591170e+00,-8.72890770e-01,-4.82534021e-01],[-5.00361621e-01,-3.55742186e-01,-5.65011024e-01],[1.23862801e-02,1.67105556e-01,-3.39211039e-02]],[[4.54127192e-01,9.58473265e-01,-5.83754003e-01],[8.05291384e-02,-1.20729990e-01,2.30556935e-01],[4.58728373e-01,8.82444009e-02,-1.53727442e-01]],[[3.54115486e-01,6.78356767e-01,-4.69687492e-01],[3.02109361e-01,7.93079197e-01,-7.02010989e-01],[6.10109091e-01,1.90677285e-01,-1.62795112e-01]],[[1.80935204e-01,7.43532404e-02,-3.66828680e-01],[-3.27710360e-02,1.17519505e-01,2.62603089e-02],[-6.17657185e-01,-1.25882000e-01,-1.09901883e-01]],[[-1.18178762e-01,5.10669470e-01,2.28295282e-01],[1.10004127e-01,-5.43532610e-01,4.60154176e-01],[-4.09266651e-01,-1.15853429e+00,1.09705317e+00]],[[3.59551281e-01,3.45320031e-02,-4.88122404e-01],[-8.06515142e-02,6.02018118e-01,-1.48388624e-01],[-8.20771605e-02,9.65389386e-02,-1.31902784e-01]],[[4.92133111e-01,-3.43066379e-02,-5.13050139e-01],[-5.44916332e-01,5.51794529e-01,-1.41229495e-01],[-1.05453849e-01,1.19537465e-01,-1.19606984e+00]],[[1.02982819e-01,-4.63730127e-01,3.60537499e-01],[1.18219905e-01,3.13819826e-01,-3.15325499e-01],[-4.67294753e-01,2.50703841e-02,-2.92603582e-01]],[[-8.63170028e-02,4.07294601e-01,6.24040179e-02],[-1.17524587e-01,-4.50931847e-01,2.86726713e-01],[-1.85112089e-01,-1.49177192e-02,8.07717144e-01]],[[-1.89582795e-01,-9.57946837e-01,-4.83128548e-01],[2.13539585e-01,-1.77804276e-01,-2.47240454e-01],[3.37530822e-01,1.10084981e-01,-3.07126850e-01]],[[8.18288386e-01,9.10810009e-02,-1.38819665e-01],[5.93856215e-01,-5.71477413e-01,1.10030627e+00],[-2.17958048e-01,-2.75235206e-01,2.02745035e-01]],[[-2.15306327e-01,-6.22453570e-01,2.59382486e-01],[1.10282591e-02,-4.69540060e-02,4.64315474e-01],[2.74587184e-01,5.26084900e-01,-7.17789650e-01]],[[6.67991221e-01,-2.81652180e-03,4.99439240e-01],[-2.86769360e-01,1.48503512e-01,1.55067081e-02],[-3.55721921e-01,-5.53642631e-01,6.78584874e-01]],[[-1.42343238e-01,2.96250403e-01,4.35197175e-01],[8.24432522e-02,-2.12184936e-01,5.46514094e-01],[-1.89344317e-01,2.87742049e-01,-1.67747632e-01]],[[-5.36592424e-01,-4.86444712e-01,-6.09220564e-01],[-3.19976628e-01,-6.83827519e-01,-7.56662413e-02],[-2.06284732e-01,-4.04770970e-01,-6.20452046e-01]],[[5.25788128e-01,5.99120140e-01,-1.51291043e-01],[-5.10162950e-01,-3.77159983e-01,-5.78789592e-01],[6.39159441e-01,-8.97446722e-02,-4.96078372e-01]],[[2.23876864e-01,9.70985591e-02,-4.96079654e-01],[5.69678307e-01,1.13600731e-01,5.80933206e-02],[-1.10026665e-01,2.19458699e-01,-1.72011331e-01]],[[-3.66530061e-01,5.17122567e-01,-4.14131284e-01],[-1.95293173e-01,3.11120510e-01,-3.37240219e-01],[2.51480013e-01,2.64348507e-01,4.20963198e-01]],[[2.66455203e-01,-1.78541884e-01,5.87482631e-01],[1.99668735e-01,-3.98807794e-01,6.27056301e-01],[1.51885390e+00,2.88635343e-01,8.16432416e-01]]],[[[4.57582176e-01,1.29402444e-01,-5.26762962e-01],[-4.19169575e-01,4.52274710e-01,-5.06752610e-01],[2.11330548e-01,-2.33489685e-02,6.39126450e-02]],[[-4.12728012e-01,-2.82404393e-01,-7.78077096e-02],[1.33526057e-01,8.49572122e-02,2.00737774e-01],[6.65013075e-01,6.32569373e-01,5.31468153e-01]],[[-2.29408488e-01,6.15508378e-01,8.68759215e-01],[2.52557278e-01,-1.13033071e-01,-8.00633430e-01],[-2.46406496e-02,-2.19498295e-02,-2.56063014e-01]],[[-4.09644872e-01,-4.10231203e-01,6.13647699e-01],[4.31160212e-01,-2.62398213e-01,2.41168112e-01],[-3.00417751e-01,1.99638695e-01,8.11773315e-02]],[[-4.53750879e-01,4.98585552e-02,7.74393618e-01],[-4.20446604e-01,-5.40946871e-02,-7.31567383e-01],[4.34357405e-01,2.37396583e-01,1.45390475e+00]],[[1.83838919e-01,5.52104898e-02,-3.06475759e-01],[3.94632593e-02,5.52824795e-01,9.19330567e-02],[4.72905517e-01,4.56673771e-01,2.49368027e-01]],[[-1.45103350e-01,6.67988598e-01,-1.72996700e-01],[-1.82926044e-01,2.57212281e-01,4.38220173e-01],[-7.86368698e-02,8.47077847e-01,-3.63427907e-01]],[[2.73550935e-02,-2.13211522e-01,8.64897370e-01],[-2.94497192e-01,9.69228208e-01,3.62651408e-01],[-5.79629481e-01,-1.36400506e-01,2.97864676e-01]],[[4.62342262e-01,-8.08457792e-01,7.67791927e-01],[-5.62778950e-01,2.44769245e-01,2.92000979e-01],[6.17544949e-01,-7.04811037e-01,-1.31760329e-01]],[[5.41665971e-01,3.35581511e-01,-4.61152792e-01],[-7.12350845e-01,6.25609338e-01,-4.77628976e-01],[2.13391274e-01,-4.79954243e-01,-1.50240406e-01]],[[-9.21194434e-01,-9.61294115e-01,-4.47626323e-01],[1.88465491e-01,-8.39908063e-01,2.92216390e-01],[-1.15825057e-01,-5.38982630e-01,-7.49641716e-01]],[[-4.43941541e-02,1.88810781e-01,9.68213603e-02],[4.04000133e-01,2.97936171e-01,3.31886798e-01],[-3.11567277e-01,2.95329064e-01,7.74503052e-02]],[[-3.95913482e-01,-5.99652052e-01,-5.86654902e-01],[-3.43298465e-01,6.86428726e-01,2.40514785e-01],[4.17340845e-01,-1.68125451e-01,-4.61904258e-01]],[[-1.93800807e-01,2.84555763e-01,-4.50313270e-01],[2.02156514e-01,-6.50631487e-01,-1.88538462e-01],[1.70229524e-01,-5.48094273e-01,1.39127433e-01]],[[-1.07990772e-01,1.05472989e-01,1.14543185e-01],[8.86986032e-02,3.55074257e-01,3.97165045e-02],[1.54775485e-01,2.32302979e-01,-4.23359960e-01]],[[3.43644135e-02,4.54134881e-01,-3.19622308e-01],[8.60017240e-01,-3.92320067e-01,-1.92005545e-01],[-4.89554405e-01,-4.44695145e-01,3.01065922e-01]],[[-3.65372747e-01,-5.36944151e-01,-1.83267772e-01],[-7.26461470e-01,9.58770514e-02,-1.84600353e-01],[7.70160139e-01,-9.70056057e-01,-3.34646136e-01]],[[5.49711883e-01,-1.63268462e-01,3.87883157e-01],[-5.32062113e-01,8.34678948e-01,-6.45014346e-01],[-2.90464789e-01,7.50553191e-01,7.54046202e-01]],[[1.35084316e-01,-4.19127613e-01,-8.77171457e-01],[-8.26720953e-01,-2.01172113e-01,-2.40717024e-01],[3.08099687e-01,9.21497822e-01,2.46350065e-01]],[[9.73321676e-01,1.87555939e-01,8.77841532e-01],[-2.07494199e-01,3.74449164e-01,7.57267535e-01],[-3.18405986e-01,-6.27104044e-02,-3.62801790e-01]],[[-8.10711980e-01,8.73308554e-02,5.37772715e-01],[1.07157409e-01,-3.72389197e-01,-9.25584733e-02],[-3.45136374e-01,2.72511542e-02,5.85577428e-01]],[[1.04849068e-02,2.07128525e-02,-1.84174344e-01],[2.76630253e-01,-4.26336050e-01,2.92507738e-01],[-1.18368931e-01,-1.11055398e+00,1.79577157e-01]],[[-2.67752677e-01,-6.03732407e-01,1.88671827e-01],[2.27605700e-01,2.65267551e-01,-1.36877242e-02],[2.54216582e-01,-5.72460070e-02,3.89334150e-02]],[[-3.55769545e-01,-9.20993626e-01,-7.36987829e-01],[3.52153853e-02,1.79771483e-01,1.87365174e-01],[-1.36878371e-01,-2.57964104e-01,1.20047443e-01]],[[2.40219489e-01,-1.39542475e-01,3.88793588e-01],[-6.94275200e-01,-8.19467902e-01,2.84817249e-01],[-1.19166479e-01,2.34227121e-01,-2.69906789e-01]],[[7.01922953e-01,-7.75707513e-02,1.41390726e-01],[1.08963323e+00,1.12813510e-01,-1.86282415e-02],[3.72649699e-01,-5.44086397e-01,5.99560976e-01]],[[-5.35365865e-02,-1.90980509e-01,-5.60405366e-02],[-7.96739101e-01,-3.91543627e-01,1.58866450e-01],[2.48457134e-01,-3.90165269e-01,1.74971431e-01]],[[4.87666816e-01,1.21753015e-01,6.87354684e-01],[1.14770725e-01,-1.86132908e-01,-5.32222867e-01],[8.70538712e-01,8.53550375e-01,-1.24716604e+00]],[[-4.13940877e-01,-2.73016930e-01,4.17349428e-01],[9.88951400e-02,-4.20949757e-01,-6.96610987e-01],[2.07646832e-01,1.09129705e-01,-1.73784330e-01]],[[-3.87190692e-02,4.08585072e-01,-3.50880101e-02],[-1.20407514e-01,-1.24091841e-01,-8.70890617e-01],[-1.13526478e-01,4.45650458e-01,1.79845504e-02]],[[-2.07057625e-01,-1.13170287e-02,-5.31254709e-01],[-2.86654532e-01,2.36625507e-01,-6.50828063e-01],[3.46888490e-02,-1.41018391e-01,-1.24355748e-01]],[[1.74244016e-01,-1.91227198e-01,-4.51010823e-01],[-6.15703285e-01,-5.50544620e-01,3.40628803e-01],[4.67760265e-02,-3.56011957e-01,9.80537906e-02]],[[6.28635168e-01,6.72488585e-02,4.68720436e-01],[-7.52810985e-02,1.01597562e-01,-7.54775405e-02],[7.29108691e-01,2.05768168e-01,-2.65762597e-01]],[[2.55883168e-02,5.63291192e-01,6.63146317e-01],[-5.32165885e-01,-1.96730480e-01,7.30978441e-04],[-2.74840184e-02,-2.67996997e-01,4.52547550e-01]],[[-2.33786091e-01,-3.99899215e-01,-2.16635913e-01],[-5.35233498e-01,-1.64684862e-01,-5.09789944e-01],[2.23246455e-01,5.89768589e-01,-9.82666969e-01]],[[8.87010917e-02,-1.37126073e-01,1.87882438e-01],[-1.10629201e+00,-2.56250203e-01,-2.13168617e-02],[3.52320910e-01,8.02017078e-02,5.39749682e-01]],[[4.57269065e-02,4.80073392e-01,7.88148344e-02],[1.37745190e+00,5.18887937e-01,-4.71038520e-01],[9.39435005e-01,-1.09834261e-01,7.51608312e-01]],[[2.24462897e-02,4.11741227e-01,2.68159285e-02],[-9.43478703e-01,-1.30819261e-01,4.34587933e-02],[1.90203011e-01,1.27548769e-01,-3.54319327e-02]],[[-2.55944014e-01,3.93430144e-01,2.02034488e-01],[2.77223259e-01,7.34083652e-01,4.44824010e-01],[-9.28488970e-02,3.21704835e-01,1.24314949e-01]],[[-5.05738735e-01,-1.76532716e-01,4.98842686e-01],[-2.81779736e-01,2.89355755e-01,-1.61669344e-01],[2.24468276e-01,-5.85556388e-01,2.61242360e-01]]],[[[0.26763886,0.6511779,0.97887594],[0.03329732,0.14578255,-0.07973714],[-0.07983239,0.5119014,0.2885126]],[[-0.3776847,0.05366802,0.04485586],[-0.262376,-0.8743256,-0.17162617],[-0.5387852,-0.47839126,-0.7690787]],[[-1.0041181,-0.51475716,-0.5026372],[0.22366115,-0.3335738,0.27225825],[0.1397206,0.12266301,-0.24602273]],[[0.03321313,-0.23574911,-0.6125456],[-0.59761304,0.1503228,0.01357131],[0.1803468,-0.22405837,-0.8455372]],[[-0.67613214,0.95968765,0.09722118],[-0.67061675,-0.7389638,-0.2842294],[0.04863388,-0.53508013,-0.19489329]],[[0.70702535,-0.3300287,-1.3523537],[-0.04773138,-1.0217378,0.7047609],[0.04857045,-0.65747285,0.06649657]],[[-1.0982105,1.0115027,0.6120424],[-0.5798755,-0.04834812,-0.03503188],[-0.16989507,0.23657241,-0.2001592]],[[0.17312826,-0.91866755,-0.2596321],[-0.13352196,-0.01089165,-0.2935969],[0.16963448,0.09043158,-0.07721401]],[[0.09537294,-0.01819632,-0.05188207],[0.8755303,-0.11279826,-0.01444225],[0.9604033,-0.07022411,-0.29244912]],[[-0.6615897,0.1681692,-0.06486471],[0.58144814,0.13545091,-0.10933699],[-0.29319513,-0.16141376,0.5008883]],[[0.6571621,-0.25378063,0.65311867],[-0.04436606,0.26727182,-0.63626444],[0.18592256,0.50968915,-0.22420329]],[[-0.9974556,-0.50314224,-0.06793859],[0.6875729,-0.6096084,0.4146788],[0.7266993,0.22100039,-0.45317876]],[[0.4664701,0.43819496,-0.0650559],[-0.2149039,-0.43055603,-0.4297309],[0.93956834,-0.47867364,0.36577716]],[[0.4225457,-0.04197233,-0.7315543],[-0.35180268,-0.03914686,0.46433392],[0.3434102,0.04949337,0.05148897]],[[0.19357798,0.0574755,0.6388248],[-0.58546436,-0.05765928,-0.13800728],[0.05847199,-1.2387892,0.40376496]],[[0.5286562,0.40270188,0.5315329],[0.49356604,-0.0465364,-0.07484046],[0.7602886,0.03205066,-0.46094462]],[[-0.1809105,-0.40675756,-0.28154486],[-0.17481945,0.26034108,0.1936534],[-0.3975325,0.06085451,0.6111785]],[[-0.31800365,0.44319496,-0.30408248],[-0.8421648,0.5143163,0.19813879],[0.57349724,0.6533151,0.7581292]],[[-0.16396487,0.15714906,-0.02070462],[-1.1896625,0.55839556,-0.05314807],[-0.21030629,-0.2624988,0.91140443]],[[-0.33403715,-0.21699123,0.00681302],[-0.01377048,0.43990585,0.19179948],[0.90749085,0.03345717,0.6647531]],[[0.62493175,-0.37332064,-0.28249484],[-0.24044043,-0.39790735,-0.37018064],[0.14346677,0.13734084,0.06566133]],[[-0.17744239,-0.01935243,0.41531062],[0.01770956,-0.073062,0.20629066],[-0.453447,-0.59938395,0.35660657]],[[0.17723846,0.5856326,1.0654738],[0.05720863,0.4767629,0.29123855],[-0.0655281,0.36495736,0.24179184]],[[-0.14669605,0.29086825,-0.37655544],[0.29866862,0.6091221,-0.06373166],[-0.13295156,-0.06952974,0.64416665]],[[0.11010166,0.49944293,-0.1471911],[-0.23733343,0.3983805,-0.0014521],[0.10911859,-0.39803666,0.18673165]],[[-0.6054447,0.26946387,0.08381061],[-0.31940472,-0.03802499,-0.04648756],[-0.3153284,0.9489647,1.1503414]],[[0.12783167,0.18129602,0.07182697],[0.35966238,-0.98343843,0.7137656],[0.03685277,-0.4837887,0.35690078]],[[0.02390638,-0.20088935,0.03698201],[-0.15755454,-0.3518849,0.5641426],[0.7875051,-0.28512174,0.24167033]],[[-0.07632129,-0.40657455,-0.14607656],[0.40325546,-0.17062642,-0.10503717],[-0.19600998,-0.5261098,-0.20739004]],[[-0.31783575,-0.07923833,-1.1367416],[-0.09560753,-0.42903882,0.02711751],[-0.3789099,-0.19711593,-0.20842591]],[[0.28729963,1.3109068,0.53122747],[-0.6657942,0.64746785,-0.05785092],[-0.3158256,0.69866616,-0.6798802]],[[-0.5057729,-0.12776186,0.30699065],[-0.44124556,0.5823587,0.01801235],[-0.3190682,1.0125984,1.2423676]],[[-0.11642927,1.3085726,0.30189285],[-0.02301798,0.65153176,-0.26287875],[-0.20344703,-0.03936158,-0.38077232]],[[0.10320347,-0.27752885,0.12448345],[0.49280363,-0.2844447,0.5263337],[0.01191617,0.5683548,0.14022242]],[[0.35681468,0.29291385,0.03244415],[0.79313564,-0.59810835,0.30944085],[-0.2168221,0.23797974,0.13946493]],[[0.4658652,-0.4374959,0.58471507],[0.46859655,0.37358347,-0.33315286],[-0.27799964,-0.4049143,0.6308567]],[[0.06526256,-0.63829255,0.7286935],[-0.25281745,0.28113583,0.5155334],[-0.8101641,-0.7527262,0.18323897]],[[-0.5996224,-1.2634312,0.49889147],[-0.16052464,-0.37512362,-0.45536432],[0.12088927,0.17589849,-0.15684077]],[[-0.10785257,-0.51841193,0.11835134],[-0.77715576,0.05268015,0.04104605],[0.7784023,-0.09656597,-0.2892032]],[[-0.4405518,-0.423515,-0.27704996],[-0.01695682,-0.04068471,0.24334441],[-0.43935287,-0.01423042,0.02189589]]],[[[-8.49009991e-01,2.30988562e-01,-1.49190098e-01],[1.88462034e-01,-7.00148344e-01,-5.52057266e-01],[5.93604445e-01,1.86801136e-01,3.36569995e-01]],[[3.79711330e-01,-4.32620913e-01,-3.91967088e-01],[4.43059742e-01,3.63562018e-01,9.11930740e-01],[-7.16220438e-01,4.42854851e-01,4.65640485e-01]],[[8.04286376e-02,-2.93138176e-01,2.17754483e-01],[1.58352256e-01,2.91288167e-01,3.78767014e-01],[-3.26291949e-01,-6.35973737e-02,-2.84293950e-01]],[[6.25403404e-01,-5.07246614e-01,-5.96545994e-01],[4.00717884e-01,-3.21641892e-01,4.93468553e-01],[4.13229406e-01,-5.28005123e-01,4.78415370e-01]],[[4.84590769e-01,-1.59963816e-02,1.74704507e-01],[-3.18059959e-02,1.75442874e-01,-1.74102798e-01],[4.59090099e-02,8.83195102e-01,-6.05131030e-01]],[[-4.07019556e-01,5.19820392e-01,3.99806619e-01],[-2.53180236e-01,5.20231843e-01,-1.52132198e-01],[1.06385248e-02,-2.29084939e-01,1.70234770e-01]],[[2.98757315e-01,-6.69077113e-02,-1.89203784e-01],[4.09729153e-01,4.57905054e-01,-3.07510227e-01],[-5.76419294e-01,1.17670171e-01,-8.28905880e-01]],[[6.05724514e-01,-4.43457544e-01,-3.99109423e-01],[-6.51910186e-01,-3.24338615e-01,4.68555897e-01],[1.15581833e-01,1.33528039e-01,-1.18960190e+00]],[[5.17579876e-02,3.22307050e-01,-1.97812542e-01],[-1.02159524e+00,6.28574863e-02,2.32800171e-02],[8.39516640e-01,-8.15858915e-02,-2.58358061e-01]],[[-3.61657441e-01,-5.38698137e-01,1.80697277e-01],[3.29136610e-01,-6.61265612e-01,-4.07298833e-01],[-2.24260047e-01,-4.67853844e-01,3.55319351e-01]],[[-1.81455567e-01,-1.24326661e-01,9.34851348e-01],[8.33860576e-01,1.00901714e-02,1.60907388e-01],[-3.42247337e-01,6.57318652e-01,-8.36196721e-01]],[[6.16216719e-01,6.84511781e-01,-2.63717115e-01],[-2.10428625e-01,-1.96881250e-01,-4.44154799e-01],[-3.51170987e-01,-4.99355584e-01,-1.28113449e+00]],[[9.31658506e-01,4.78129178e-01,-6.26587048e-02],[-8.35353732e-02,3.63113016e-01,-3.78665864e-01],[-5.81131339e-01,7.44716600e-02,-3.54632251e-02]],[[1.58026084e-01,-1.46892726e-01,1.93708643e-01],[9.54870228e-03,-2.42682755e-01,3.12127590e-01],[-5.24954855e-01,-9.53063130e-01,1.41557276e-01]],[[-1.83337823e-01,8.81846994e-02,1.17960699e-01],[-4.10003662e-01,6.76702321e-01,-3.41033369e-01],[-1.27138400e+00,3.53423297e-01,-8.45299363e-02]],[[1.76747441e-01,-8.08850229e-02,-3.62186670e-01],[-6.68157870e-03,-1.80387616e-01,-3.85887295e-01],[2.33252078e-01,-3.46813933e-04,6.10375702e-01]],[[5.53714454e-01,-3.46415251e-01,-2.21299008e-01],[1.12818383e-01,-2.65631199e-01,-5.03554702e-01],[3.97753447e-01,2.75492489e-01,-8.43564212e-01]],[[5.93286268e-02,-2.29263201e-01,7.83416554e-02],[-1.73844993e-01,-3.41099113e-01,2.08530799e-01],[5.49831867e-01,-6.51133657e-01,-8.25534880e-01]],[[-6.12894259e-02,6.57517672e-01,2.16707349e-01],[3.63483936e-01,8.04200888e-01,-7.90751204e-02],[-3.01604062e-01,-3.31397466e-02,1.25176832e-01]],[[2.30037302e-01,-2.04607874e-01,2.17257470e-01],[4.23780739e-01,6.07036352e-01,-2.35348120e-01],[-4.71273204e-03,6.20930642e-02,-6.67701304e-01]],[[-9.61669758e-02,-1.49775967e-01,-9.45485234e-02],[-1.34878263e-01,2.35905454e-01,-2.12569386e-01],[5.22218108e-01,5.03491104e-01,6.31415546e-01]],[[5.44565082e-01,3.97641957e-01,-4.18889761e-01],[1.32889869e-02,-2.64849275e-01,-5.52874267e-01],[5.26018403e-02,-3.36936116e-01,-2.59455293e-01]],[[-4.71516937e-01,-1.38222039e-01,-4.22722906e-01],[2.39236698e-01,5.03802486e-02,-5.24984062e-01],[5.42671502e-01,-5.19056559e-01,-5.64426959e-01]],[[4.87862796e-01,-2.45633751e-01,-2.61356413e-01],[-7.57619798e-01,2.15827942e-01,6.86603040e-02],[5.31702861e-02,-6.50952339e-01,1.18085198e-01]],[[3.73681068e-01,-2.49659419e-01,-2.70264387e-01],[6.69753790e-01,-4.09926295e-01,-3.01647425e-01],[1.78426534e-01,-6.36390746e-01,7.86191523e-01]],[[-2.20613882e-01,2.45797515e-01,3.80143106e-01],[3.86177227e-02,2.90392309e-01,8.18832219e-02],[1.92215115e-01,5.09123921e-01,-7.10841894e-01]],[[2.11514965e-01,-1.70616254e-01,-5.96983135e-02],[7.45278075e-02,-3.89562845e-01,-6.27040207e-01],[-9.31065604e-02,7.15238929e-01,-8.03527236e-01]],[[-3.82148623e-01,1.65061906e-01,6.60670102e-01],[-1.33627623e-01,-1.60716444e-01,1.56422466e-01],[-1.55626035e+00,-5.04434817e-02,-2.08670273e-01]],[[-3.78906310e-01,1.50971860e-01,3.49174410e-01],[-2.11487442e-01,-2.74424464e-01,3.69696051e-01],[2.63880976e-02,4.74816859e-01,3.31847370e-01]],[[4.12106887e-02,-5.74278057e-01,-7.19241560e-01],[-7.39842802e-02,-4.70181465e-01,-5.99051237e-01],[9.67803717e-01,-1.57661033e+00,-9.54484582e-01]],[[-1.90634385e-01,-1.42890988e-02,7.47645646e-02],[9.81495798e-01,3.52357626e-01,6.90518618e-01],[-1.99197486e-01,-1.12280238e+00,-3.63534153e-01]],[[5.00917614e-01,1.72979400e-01,4.04023945e-01],[-2.48670563e-01,-3.55278373e-01,1.45929664e-01],[-1.13912567e-01,6.71263877e-03,1.15600094e-01]],[[1.94460586e-01,-4.11470503e-01,-1.91135794e-01],[-2.12612152e-01,1.27673477e-01,-2.95284092e-01],[1.90250456e-01,8.09883848e-02,1.62949741e-01]],[[7.11385369e-01,5.36630750e-01,-1.62089482e-01],[-4.62877095e-01,2.69071966e-01,-2.78625358e-02],[-1.49267837e-01,1.98944613e-01,1.40177548e-01]],[[-9.09342021e-02,-3.24567676e-01,-3.17742154e-02],[-1.20541489e+00,-5.09661257e-01,-1.82714999e-01],[5.99589467e-01,-7.96631277e-01,-1.00350058e+00]],[[1.61850676e-01,-1.26813143e-01,2.50642728e-02],[8.56989846e-02,-2.79866457e-01,2.26955533e-01],[1.09066081e+00,-4.26408827e-01,-3.70062739e-02]],[[2.46553600e-01,2.40255028e-01,6.01820014e-02],[-2.32167885e-01,1.47285357e-01,4.63592112e-01],[-5.56000620e-02,-5.88885248e-01,-6.88497782e-01]],[[-4.37837690e-01,2.86732204e-02,-4.79172796e-01],[-3.03054690e-01,-2.24055409e-01,-3.67443711e-01],[-1.17485797e+00,-7.36963674e-02,-5.15965283e-01]],[[2.98787504e-01,-8.06182846e-02,-1.83830261e-01],[-3.91037583e-01,-7.48702288e-01,3.56931910e-02],[1.66045353e-01,2.40836591e-01,1.35976166e-01]],[[4.63809162e-01,-5.79979382e-02,-2.81597704e-01],[8.87094259e-01,3.99974167e-01,1.05895422e-01],[4.02865827e-01,5.65764681e-03,4.55698520e-01]]],[[[-6.57804787e-01,4.07693148e-01,-1.37043968e-01],[-1.12243772e+00,-3.41684252e-01,-5.24464428e-01],[2.61477455e-02,-1.26766279e-01,9.97436941e-02]],[[7.61172533e-01,4.48499143e-01,-2.02222705e-01],[6.19118631e-01,-2.35525548e-01,-1.42669991e-01],[-2.43560914e-02,2.73529589e-01,3.65165621e-01]],[[3.01558703e-01,1.69488072e-01,4.52591807e-01],[-4.86592591e-01,-6.68296695e-01,3.01704705e-01],[-6.50372624e-01,6.76787794e-01,1.76960230e-02]],[[-6.52797103e-01,1.61238208e-01,6.09548151e-01],[-7.45694935e-01,-4.41536605e-02,-2.78002858e-01],[4.49612856e-01,-3.99275184e-01,-7.82919645e-01]],[[-3.44114423e-01,7.62858331e-01,-1.34828079e+00],[-7.67433465e-01,-3.74099940e-01,-1.21936709e-01],[1.11956690e-02,9.51281413e-02,-3.09534132e-01]],[[-1.99702889e-01,2.12463140e-01,3.75597253e-02],[8.00266191e-02,6.02319315e-02,3.42913806e-01],[1.95311867e-02,6.05869770e-01,5.71626961e-01]],[[-1.26744300e-01,-4.80241746e-01,-1.14545882e+00],[3.77417445e-01,-7.16227412e-01,-6.42451584e-01],[1.03384173e+00,5.56086972e-02,1.03449017e-01]],[[8.00156891e-01,-1.50405392e-01,-4.66017518e-03],[-2.98301965e-01,-4.61758256e-01,6.45509958e-01],[6.09280944e-01,-4.46910739e-01,-6.37277663e-02]],[[-5.89075863e-01,1.06666458e+00,-2.57314414e-01],[3.69728506e-01,4.48507816e-01,-3.83147866e-01],[5.04318655e-01,3.33864801e-03,2.50561908e-02]],[[7.27252483e-01,-4.18662339e-01,-5.45982659e-01],[2.30250806e-01,-2.50235826e-01,-5.35825361e-03],[3.86441976e-01,-6.33505592e-03,7.78778255e-01]],[[1.10957421e-01,1.55639827e-01,-7.98323095e-01],[-2.12841675e-01,1.55502498e-01,2.75780894e-02],[1.58522993e-01,2.70150434e-02,5.61499119e-01]],[[5.29944360e-01,-2.33183168e-02,-1.66174360e-02],[1.11408949e+00,2.10618377e-01,-2.61532456e-01],[-3.17357928e-02,4.17871475e-01,4.75918889e-01]],[[-9.61357176e-01,-8.26681793e-01,-2.56083250e-01],[-4.37469691e-01,1.80261046e-01,-1.89196751e-01],[-1.94486246e-01,2.76226282e-01,-7.36460268e-01]],[[9.50332806e-02,5.15947402e-01,1.61625534e-01],[-1.36801749e-01,7.57094026e-02,2.44161971e-02],[4.27048177e-01,2.90083170e-01,-1.52279630e-01]],[[-2.14330498e-02,-4.94491100e-01,-5.95568538e-01],[-6.15993679e-01,3.18093956e-01,3.86631995e-01],[-3.06135744e-01,-1.32350810e-02,-3.74855101e-01]],[[-1.59161419e-01,-2.91365564e-01,-3.32910508e-01],[2.83505797e-01,-3.00701469e-01,-5.12390375e-01],[3.74159038e-01,1.63122505e-01,-1.63653359e-01]],[[7.43893147e-01,3.07213217e-01,1.28727004e-01],[9.80483413e-01,-2.08788738e-01,-7.48173356e-01],[-3.40934873e-01,-1.55850723e-01,-1.16007650e+00]],[[-7.17786431e-01,-1.23876423e-01,-2.73889005e-01],[-5.63868396e-02,9.45240378e-01,1.65599361e-01],[-7.28892028e-01,-4.57870275e-01,8.99847075e-02]],[[2.75440186e-01,2.01632157e-01,-1.03415318e-01],[2.27331728e-01,-1.96201622e-01,-4.50188927e-02],[1.41023427e-01,2.77683169e-01,4.91717070e-01]],[[4.08286422e-01,-2.36996129e-01,2.02942118e-01],[4.92327034e-01,-3.18036415e-02,-6.24543801e-02],[4.96939361e-01,-2.22612515e-01,1.17837533e-01]],[[-7.27547586e-01,-5.81478775e-01,-2.33133778e-01],[-4.97159719e-01,1.75853521e-01,-4.45856303e-02],[3.40657346e-02,3.54515910e-01,7.29887336e-02]],[[-1.36925932e-02,1.67085707e-01,-2.97700077e-01],[3.15730810e-01,-2.68296510e-01,-1.76492438e-01],[-4.33263540e-01,-2.24062711e-01,2.10971683e-01]],[[3.83389860e-01,4.22925055e-01,7.38104880e-01],[2.76803583e-01,5.81901610e-01,-1.86302483e-01],[5.74877858e-03,-1.57829002e-01,4.93871689e-01]],[[2.54822701e-01,-4.25983071e-02,-6.86454892e-01],[7.28162885e-01,1.76408798e-01,3.26770395e-01],[3.56402099e-01,2.15100840e-01,-6.65043235e-01]],[[6.15889728e-01,1.60391718e-01,1.39887244e-01],[9.62663233e-01,1.72680154e-01,-2.32232109e-01],[2.45693624e-01,-2.00307727e-01,1.08062434e+00]],[[1.98619545e-01,1.90739453e-01,-4.92500484e-01],[3.75838637e-01,-8.96550059e-01,-5.03401518e-01],[5.70233047e-01,-3.20549190e-01,2.50441611e-01]],[[-4.18780409e-02,-8.68277103e-02,-2.03991994e-01],[-6.03754446e-02,1.39822233e-02,-2.86368489e-01],[8.88688922e-01,2.03775004e-01,1.12921439e-01]],[[8.02334964e-01,-5.08947372e-01,2.54529089e-01],[-3.12259346e-02,9.19405073e-02,6.08950675e-01],[4.71212208e-01,1.40460297e-01,-9.44167525e-02]],[[2.10567504e-01,-7.00607479e-01,-1.16442573e+00],[4.15704370e-01,1.62499696e-01,-1.99762136e-01],[-1.86105385e-01,-3.89291346e-01,-5.52646279e-01]],[[-3.23023319e-01,9.26735938e-01,9.74829853e-01],[-4.11048740e-01,1.12370956e+00,1.74721733e-01],[-5.62543273e-01,3.05550814e-01,-2.83001751e-01]],[[-5.99852979e-01,-4.51171666e-01,-3.51523936e-01],[-2.98056066e-01,-1.11574911e-01,-3.12845141e-01],[-2.61841826e-02,5.55656016e-01,-1.17197514e-01]],[[5.05694211e-01,-2.92407840e-01,-9.00313139e-01],[2.49048993e-01,3.60226870e-01,-3.15981776e-01],[1.72663957e-01,-2.02598751e-01,-1.23812187e+00]],[[8.26327875e-02,5.04338980e-01,1.95127979e-01],[2.16626585e-01,-2.54388630e-01,8.66261646e-02],[3.22558105e-01,6.26519561e-01,-3.70729297e-01]],[[-2.86596537e-01,-4.08798642e-02,4.24000472e-01],[2.42986735e-02,5.99178255e-01,-1.54648781e-01],[2.46077523e-01,3.85799080e-01,5.29025614e-01]],[[1.45311832e-01,2.04482973e-01,-7.10345507e-01],[4.55516219e-01,3.70592512e-02,2.28812117e-02],[-3.72963101e-01,-1.79076672e-01,-7.50015199e-01]],[[-1.97722286e-01,-4.04960252e-02,6.17664278e-01],[5.62312722e-01,3.22298676e-01,-7.81944022e-02],[4.33714628e-01,-1.45833731e-01,5.42641580e-01]],[[2.38472208e-01,1.12530100e-03,-2.62383848e-01],[4.64631259e-01,7.34662592e-01,4.47235078e-01],[-1.80744052e-01,7.99055159e-01,2.92785585e-01]],[[-5.21546364e-01,8.44206810e-01,-9.46551561e-02],[1.72479719e-01,8.06128457e-02,3.89261454e-01],[5.51029027e-01,-3.25581342e-01,-5.39316773e-01]],[[-1.10477161e+00,-1.04401864e-01,-2.40195617e-02],[-3.44693601e-01,4.78909388e-02,1.30329713e-01],[5.21244109e-01,-2.87414581e-01,-2.66256273e-01]],[[1.50822207e-01,1.93248063e-01,-5.29629409e-01],[5.88617861e-01,-6.61164522e-02,-9.07741785e-02],[-2.28026420e-01,3.20595741e-01,6.66308799e-04]]],[[[-3.37565839e-01,7.93906987e-01,5.90828285e-02],[3.95042375e-02,-1.96274638e-01,-3.45248938e-01],[-3.13627869e-01,3.91516834e-01,-1.17484830e-01]],[[7.71979868e-01,-2.45672360e-01,-1.63119438e-03],[7.43115366e-01,-4.72110838e-01,-4.59637642e-01],[-9.80758071e-01,-5.73195398e-01,-3.90879065e-01]],[[-2.04318296e-02,-2.85845455e-02,-1.05225116e-01],[2.79633373e-01,4.30614889e-01,-1.97200794e-02],[-1.72110885e-01,8.68020654e-02,-3.57553095e-01]],[[-6.12388909e-01,3.78067084e-02,2.63709635e-01],[-6.81569040e-01,4.59885113e-02,-5.09685874e-01],[-5.01710296e-01,-8.50882709e-01,4.02536720e-01]],[[1.53893545e-01,4.96640086e-01,1.98903278e-01],[1.24192305e-01,-2.73038089e-01,4.16180566e-02],[-2.47593284e-01,3.30062240e-01,6.01962388e-01]],[[-3.32151264e-01,-8.05430353e-01,5.02423644e-01],[-8.62587035e-01,7.01487005e-01,-2.61184096e-01],[-6.70042574e-01,-3.79110396e-01,-1.20075798e+00]],[[-7.96420753e-01,-1.39032051e-01,-7.88238719e-02],[-6.60157204e-01,-7.06281364e-01,1.04767132e+00],[3.57446223e-01,-3.06217134e-01,6.66099608e-01]],[[-5.18476188e-01,-5.22516780e-02,-4.96258557e-01],[3.56380939e-01,-6.95227146e-01,4.17466789e-01],[6.78669691e-01,3.16693075e-02,5.77053607e-01]],[[2.73619354e-01,5.87150812e-01,-3.12135488e-01],[2.18583688e-01,1.15232378e-01,3.81703749e-02],[-8.42875063e-01,1.53469844e-02,6.12566888e-01]],[[6.66153491e-01,4.67836618e-01,-5.94764352e-02],[-4.87681888e-02,-2.90538371e-01,-5.10903656e-01],[-4.92349640e-02,-6.32086575e-01,-2.61578560e-02]],[[1.58809334e-01,-5.69832504e-01,-6.50389433e-01],[8.78853977e-01,2.34865755e-01,-4.03905123e-01],[-5.68420887e-01,-1.14737138e-01,1.32226303e-01]],[[2.36117825e-01,2.08553091e-01,5.96683741e-01],[4.89200443e-01,5.86559437e-02,-5.01265645e-01],[3.84135932e-01,1.50328696e-01,3.85144114e-01]],[[-2.38161236e-01,4.82939184e-01,5.94091462e-03],[1.91734448e-01,9.11907554e-01,3.35652620e-01],[2.34000862e-01,-3.95821571e-01,-8.39537904e-02]],[[-1.00171030e-01,1.14364766e-01,1.59098238e-01],[-3.88981491e-01,-2.70626098e-01,-1.60055056e-01],[4.25683230e-01,-8.08123171e-01,-7.33857080e-02]],[[2.38333851e-01,-8.09170082e-02,-1.36517838e-01],[2.72099286e-01,-1.76547706e-01,-2.31740564e-01],[4.02334958e-01,2.17236638e-01,5.15343308e-01]],[[3.19198072e-01,7.91887641e-01,6.80987060e-01],[3.28516930e-01,-3.11567336e-01,-5.21754384e-01],[-4.95272249e-01,-1.45497635e-01,5.73118091e-01]],[[3.13048780e-01,-3.47336113e-01,5.21341145e-01],[4.77365613e-01,1.61371276e-01,-2.15451807e-01],[-2.24541977e-01,-2.86926806e-01,2.21077934e-01]],[[-6.09026194e-01,-1.35805607e-01,3.11466396e-01],[-1.81112245e-01,5.69394052e-01,-6.37742937e-01],[-5.01683280e-02,7.28935897e-01,6.00410700e-01]],[[4.76980321e-02,-5.56380332e-01,-2.34763727e-01],[-2.17807777e-02,9.42033410e-01,-2.85446793e-01],[6.51800752e-01,-4.22405809e-01,-6.15644634e-01]],[[3.97812903e-01,-3.72594595e-01,1.52296752e-01],[-1.03663176e-01,3.87555733e-02,-2.89207309e-01],[1.09228216e-01,2.03476816e-01,-1.02193165e+00]],[[-1.44435510e-01,-1.75540835e-01,1.37475038e+00],[2.37601221e-01,-2.68713921e-01,4.99755859e-01],[9.70612317e-02,3.06059182e-01,-5.67645669e-01]],[[-9.75317001e-01,-2.02327371e-01,2.90563494e-01],[-3.75509918e-01,-8.25534854e-03,2.19545484e-01],[2.01165825e-01,-1.26987532e-01,1.80885732e-01]],[[-7.41753876e-01,-5.59700578e-02,-1.64833117e+00],[-3.00841570e-01,5.22028983e-01,-1.23178206e-01],[3.26768935e-01,3.58786911e-01,-4.37911928e-01]],[[1.88750640e-01,3.92221242e-01,3.08875769e-01],[-2.88647294e-01,3.65437031e-01,-2.80925483e-01],[-5.00989676e-01,5.08562684e-01,-1.91485211e-01]],[[-5.14906347e-01,-3.14928353e-01,-5.95041692e-01],[-8.23917240e-02,-5.84200248e-02,-5.85461974e-01],[-6.92674592e-02,-3.91077191e-01,-3.15491199e-01]],[[5.33636920e-02,-2.90819198e-01,7.63516247e-01],[-1.88925415e-01,-1.00392386e-01,2.47059286e-01],[-3.72917235e-01,6.49006188e-01,-1.91864036e-02]],[[4.66050506e-02,1.78230435e-01,-7.29757667e-01],[-7.66166508e-01,-4.52272594e-01,6.29489720e-02],[-1.12206101e-01,-2.64437973e-01,-8.86186838e-01]],[[-3.92251998e-01,-2.78432434e-03,-3.31785679e-02],[-2.53795385e-01,-5.30553341e-01,2.11241812e-01],[-1.80463403e-01,1.22033924e-01,-6.69860423e-01]],[[1.91682681e-01,-4.53031451e-01,1.20605302e+00],[-3.00791740e-01,4.10956174e-01,7.17416704e-02],[2.69892402e-02,-8.27115893e-01,-5.17643929e-01]],[[-4.29152697e-01,-6.87101707e-02,-1.05803832e-01],[-5.37251711e-01,-2.37286493e-01,3.91745776e-01],[-3.45718294e-01,5.93284378e-03,-7.96196014e-02]],[[9.27535057e-01,2.63496011e-01,-2.42836609e-01],[1.33359715e-01,-1.85151864e-02,1.27909765e-01],[3.54643822e-01,-1.50149345e-01,-5.91056794e-03]],[[6.31860912e-01,-5.08129418e-01,6.37548566e-01],[4.42802101e-01,-6.05476499e-01,-4.67327327e-01],[-5.61934352e-01,-3.97662550e-01,2.01133713e-01]],[[-9.34379622e-02,2.99789250e-01,1.67226329e-01],[-4.17741761e-02,4.47791815e-01,-1.42730594e+00],[6.84591353e-01,-1.73135772e-01,1.99086070e-01]],[[-4.03523684e-01,1.05146483e-01,-1.05014555e-01],[4.91693318e-01,-7.35568479e-02,3.65738571e-01],[6.99048996e-01,-2.88953513e-01,-2.04046503e-01]],[[-5.56986690e-01,-5.41509688e-01,7.52424657e-01],[-1.33946463e-01,-4.79665995e-01,1.61941037e-01],[-4.82702732e-01,3.69866192e-01,9.02378857e-01]],[[-1.02171826e+00,6.49851501e-01,-5.83144188e-01],[3.77806097e-01,1.01820946e-01,1.00061429e+00],[-6.31565392e-01,2.60500222e-01,3.61556768e-01]],[[4.75056082e-01,-2.88693994e-01,-6.24328315e-01],[4.59932834e-01,6.94303095e-01,3.76470745e-01],[2.12777227e-01,-2.36817017e-01,2.03577742e-01]],[[-4.46999967e-01,3.29941779e-01,2.97205269e-01],[1.67567953e-01,-2.54553705e-01,1.04495682e-01],[-9.45960104e-01,-2.77843140e-02,-3.26928385e-02]],[[1.26428068e-01,5.10769308e-01,1.50768429e-01],[-1.39756173e-01,-6.11221930e-03,-5.12289941e-01],[2.14049041e-01,4.28608395e-02,-1.75182745e-01]],[[-2.07247153e-01,2.74501771e-01,-1.52253345e-01],[4.11151409e-01,-5.35585105e-01,4.80611145e-01],[-1.96785256e-01,5.17107487e-01,1.17273286e-01]]],[[[-8.61396134e-01,6.33619055e-02,-7.99457014e-01],[-6.36590660e-01,1.57719389e-01,-1.91390753e-01],[-3.38266492e-01,2.15150535e-01,-4.08944547e-01]],[[2.14635104e-01,-2.16247603e-01,-4.45887625e-01],[-2.94178903e-01,-2.09347382e-01,3.65704149e-01],[-1.34728193e-01,8.19060147e-01,8.85199532e-02]],[[-2.12558433e-01,1.35391742e-01,5.21473633e-03],[2.25224108e-01,-4.58967328e-01,-2.97750086e-02],[-4.23101872e-01,-7.45571077e-01,-1.66830689e-01]],[[-7.31562436e-01,-1.69630408e-01,5.80844223e-01],[4.15926218e-01,-1.11057329e+00,-7.20025599e-01],[9.74845514e-02,-6.92009255e-02,-1.86725520e-02]],[[-6.52059972e-01,-7.40488410e-01,-1.66650251e-01],[2.45410010e-01,-1.51866958e-01,-2.96076745e-01],[-9.52592671e-01,2.90997028e-01,-7.16000617e-01]],[[-3.05645391e-02,4.77023512e-01,-1.00301886e+00],[-2.30525017e-01,-5.99725962e-01,2.50171632e-01],[-2.75122434e-01,4.55493450e-01,6.58961982e-02]],[[-1.24646150e-01,2.90192395e-01,7.47712970e-01],[2.66016573e-01,-3.91611047e-02,-2.20918003e-02],[-2.04462573e-01,6.20628297e-01,3.00396174e-01]],[[6.14156961e-01,2.11909398e-01,3.77777755e-01],[-1.09590781e+00,2.89268702e-01,4.57668722e-01],[-3.71634036e-01,2.47563943e-01,4.24654722e-01]],[[-1.84223861e-01,-5.73153079e-01,-3.15310448e-01],[1.74615592e-01,4.04965281e-01,7.70766288e-02],[7.89113998e-01,-4.54810858e-01,2.01646596e-01]],[[5.12532592e-01,-8.49265039e-01,4.09667760e-01],[-2.25522235e-01,-2.89955258e-01,4.54250842e-01],[8.97898674e-02,3.30637634e-01,1.04819810e+00]],[[-5.50262988e-01,-8.17262113e-01,1.94544286e-01],[1.74691722e-01,-5.71493328e-01,-3.54029499e-02],[-5.99134922e-01,-5.92321455e-01,-5.04381716e-01]],[[-1.97654217e-01,-3.74797210e-02,-3.07240844e-01],[-4.29360047e-02,-3.21709514e-01,6.47181690e-01],[4.52755004e-01,-1.44975424e-01,-1.05193205e-01]],[[-1.38934091e-01,-3.14583182e-02,1.08082928e-01],[-5.10722041e-01,2.04379186e-02,-3.96948695e-01],[-2.12077335e-01,-1.53765619e-01,1.33760691e-01]],[[2.73102015e-01,5.85849643e-01,8.23618889e-01],[-5.49392402e-01,7.62917399e-02,-1.08287968e-01],[-1.37744769e-01,2.28780001e-01,-3.68731320e-02]],[[-6.22918487e-01,-5.21516614e-03,-1.91585407e-01],[-6.79115281e-02,-3.29838485e-01,1.69720963e-01],[-3.66070807e-01,7.75639713e-01,-6.28495812e-01]],[[1.76350679e-02,-4.46105301e-01,3.74976128e-01],[-6.67748332e-01,5.29242028e-03,1.45991191e-01],[4.43235099e-01,-1.82746947e-01,1.83935508e-01]],[[-3.55166316e-01,1.96059719e-01,1.24863088e-01],[1.26628757e-01,-6.94705069e-01,3.79317582e-01],[1.41323388e-01,3.66223067e-01,2.10639015e-01]],[[-2.35145073e-02,1.42173856e-01,8.29923511e-01],[-2.59555012e-01,-2.75129497e-01,-4.67790756e-03],[-3.61902505e-01,-5.87668598e-01,3.42378974e-01]],[[3.03584516e-01,-6.30813390e-02,1.75487190e-01],[-3.73927534e-01,6.78898692e-01,-2.41394833e-01],[8.01420450e-01,2.19546884e-01,1.31863132e-01]],[[-5.99273086e-01,1.43540755e-01,-2.76647568e-01],[3.04126628e-02,-1.46762118e-01,6.85348034e-01],[-1.78299025e-01,-2.29060650e-01,1.07070780e+00]],[[-3.53362441e-01,-1.38064981e-01,-3.59634519e-01],[2.92653352e-01,3.31081718e-01,4.86284494e-01],[4.87484425e-01,4.08096835e-02,-5.25972247e-01]],[[1.15387201e+00,4.09274727e-01,3.71910721e-01],[-2.22319067e-01,-1.09527826e-01,6.15124881e-01],[6.83663726e-01,1.38802260e-01,4.53787446e-02]],[[3.42893600e-01,5.84564097e-02,1.49523437e+00],[-6.40504882e-02,6.19645193e-02,5.76145649e-01],[-1.90176666e-01,1.85710285e-02,-4.60829288e-02]],[[-8.30784559e-01,-8.71158898e-01,-3.34356688e-02],[1.92952737e-01,-3.38670254e-01,-6.15210533e-02],[-1.92967191e-01,-2.41544440e-01,-1.32397294e-01]],[[8.69362354e-02,2.13565364e-01,6.40017569e-01],[1.20707564e-01,1.53464854e-01,6.45160913e-01],[5.03385186e-01,7.54136443e-02,1.15356517e+00]],[[2.22745836e-01,3.87321562e-02,-5.04328489e-01],[7.20138311e-01,2.61765063e-01,-5.34059346e-01],[-1.10757387e+00,-1.41330689e-01,2.22971797e-01]],[[-2.69206852e-01,-6.71587408e-01,9.42924544e-02],[3.98290813e-01,-4.05757993e-01,2.89996982e-01],[6.82493523e-02,8.31200331e-02,9.84567761e-01]],[[-4.94286984e-01,3.97428006e-01,5.97007573e-01],[2.72436470e-01,5.04170507e-02,7.37978399e-01],[-3.46663445e-01,-2.71601081e-02,2.98356414e-01]],[[-1.55291870e-01,-9.38024640e-01,-6.54410005e-01],[-3.16197783e-01,3.85471955e-02,-5.37080586e-01],[-3.36908065e-02,5.14475226e-01,-3.79364729e-01]],[[-3.99783999e-01,-6.18197396e-02,-8.16246420e-02],[1.52743757e-01,4.31486785e-01,-6.38150424e-02],[-9.84423697e-01,1.61224082e-01,-5.91942191e-01]],[[6.88009560e-01,4.21754569e-01,1.79132059e-01],[-3.80598187e-01,1.62390396e-01,-6.30657449e-02],[6.70072079e-01,-1.81091903e-03,-3.63130212e-01]],[[9.85499740e-01,-5.39641619e-01,4.37098965e-02],[-1.40108272e-01,-3.64222303e-02,1.46377999e-02],[7.37094820e-01,-7.02159762e-01,-3.38438421e-01]],[[-3.76976818e-01,-3.66665050e-02,3.37582916e-01],[3.42462838e-01,1.45402268e-01,2.38914356e-01],[-3.25847447e-01,-5.88996597e-02,4.45767283e-01]],[[4.62855577e-01,2.92137384e-01,5.46863377e-01],[-2.35398114e-01,-5.68222590e-02,3.47094953e-01],[1.30289882e-01,1.38446257e-01,3.11825365e-01]],[[1.14881031e-01,1.51732117e-01,-8.69779348e-01],[1.07253633e-01,3.59226704e-01,7.57569969e-01],[4.99663204e-01,-3.47587138e-01,5.55654755e-04]],[[-8.39619040e-02,1.77122317e-02,-1.09151471e+00],[1.04386315e-01,-5.78091562e-01,-4.47500139e-01],[-3.01610619e-01,-3.56294721e-01,-3.22378099e-01]],[[-8.38382840e-01,4.09056023e-02,5.18711030e-01],[1.62799522e-01,-3.16779286e-01,-3.16482335e-01],[5.82611024e-01,4.22971398e-01,-6.82006717e-01]],[[4.38556135e-01,9.96959880e-02,9.83592093e-01],[2.32463583e-01,6.96873724e-01,2.44736254e-01],[1.94744375e-02,6.30759299e-01,3.46793473e-01]],[[-7.00249076e-01,-2.75086015e-01,-5.33652842e-01],[2.79216588e-01,3.35647345e-01,2.16399238e-01],[-8.31390321e-01,-2.61928290e-01,-7.48649612e-02]],[[-5.29693440e-02,6.37489200e-01,-1.85094774e-01],[-5.00599444e-01,-1.53769523e-01,3.31415385e-01],[-3.85372818e-01,-5.47576189e-01,-3.31171742e-03]]],[[[3.30873519e-01,-5.27257085e-01,5.33628762e-02],[-3.45546305e-01,-1.86498567e-01,-6.43488884e-01],[-2.79123724e-01,8.88097063e-02,-1.39778078e-01]],[[6.32151663e-01,-1.55506596e-01,-3.58647853e-01],[2.58840859e-01,4.53756928e-01,-5.57656646e-01],[-1.61056221e-01,-7.67963767e-01,-3.09002966e-01]],[[-9.37769234e-01,7.56292403e-01,4.22926217e-01],[-9.07318354e-01,-1.14332028e-01,4.42723691e-01],[-1.88342333e-01,-2.20098630e-01,3.02163899e-01]],[[4.25352484e-01,1.46455526e-01,5.86403072e-01],[-6.64756000e-01,-1.76490620e-01,9.14694905e-01],[8.80869031e-02,2.69091666e-01,3.69702369e-01]],[[-6.94633901e-01,5.70048749e-01,2.34278083e-01],[5.51664293e-01,-1.15709566e-01,-2.40173340e-01],[2.95263827e-01,2.74451196e-01,-1.44087672e+00]],[[-2.96963036e-01,-4.17668402e-01,-4.15451765e-01],[-6.48579419e-01,1.33881748e-01,-1.22974291e-01],[-6.94531381e-01,2.92375393e-04,6.47464216e-01]],[[3.71220708e-01,3.31809610e-01,-2.23602429e-01],[1.68744311e-01,-2.98974160e-02,-4.11866695e-01],[-3.43497932e-01,1.10181704e-01,-2.67765485e-02]],[[2.34856710e-01,3.13732892e-01,2.78079156e-02],[2.97326982e-01,4.65725094e-01,5.22501349e-01],[-6.03158832e-01,-4.23767865e-01,-9.40541446e-01]],[[-9.90421996e-02,-2.88400620e-01,-9.07869041e-02],[-2.65971631e-01,2.75293350e-01,4.09031183e-01],[-3.02080840e-01,7.04063952e-01,-4.34344411e-01]],[[1.18198790e-01,-6.55381978e-01,-8.31571072e-02],[-4.63987589e-01,2.50503868e-01,3.46807897e-01],[-1.61222056e-01,5.09764969e-01,-2.53681540e-01]],[[8.64883721e-01,5.70381880e-01,2.10013896e-01],[-2.75152177e-01,-3.41561586e-01,-5.65566123e-01],[-2.50123978e-01,6.02276862e-01,-5.51230758e-02]],[[-3.25211406e-01,1.57652140e-01,7.31773257e-01],[-7.20485330e-01,1.07122147e+00,1.80941179e-01],[6.00979030e-01,-1.24200970e-01,7.53197372e-01]],[[-2.99599636e-02,-2.74297083e-03,1.86853915e-01],[4.70981419e-01,3.67035717e-01,-8.71801615e-01],[-1.44966304e-01,-2.29775250e-01,9.28700063e-03]],[[-3.87571514e-01,-6.65592775e-02,-4.10418540e-01],[-5.19118905e-01,-1.69568419e-01,-3.94454867e-01],[6.03374004e-01,-1.08823955e+00,5.08599281e-01]],[[-5.87580025e-01,-1.67212352e-01,-2.03331858e-01],[-1.79363474e-01,-4.25015002e-01,-3.48132938e-01],[5.77439703e-02,-5.75385153e-01,-3.49596977e-01]],[[1.82160273e-01,9.34257805e-01,-5.97474635e-01],[1.06858023e-01,3.78412679e-02,-3.62595975e-01],[1.07430361e-01,2.13443413e-01,-7.62404144e-01]],[[1.54513776e-01,-3.45710441e-02,-1.07537352e-01],[-2.82335907e-01,3.56414884e-01,-4.64536279e-01],[1.50692239e-01,-3.97237003e-01,-1.15879679e+00]],[[-5.35307825e-01,-1.31238341e-01,-2.90197939e-01],[-2.32981294e-02,-6.04648935e-03,-8.45310986e-02],[5.79447210e-01,1.26951143e-01,1.95848346e-02]],[[2.90453076e-01,3.70793074e-01,-5.86703718e-01],[2.06149593e-02,-1.27163398e+00,1.68322757e-01],[-3.80777121e-02,-9.98825729e-01,5.40875047e-02]],[[3.47623467e-01,3.44443500e-01,4.56964344e-01],[3.58733058e-01,3.02478492e-01,7.55858943e-02],[4.64792252e-01,-7.83788621e-01,4.83653545e-01]],[[-4.77379501e-01,3.82864475e-01,-1.96161404e-01],[-3.81113067e-02,-3.29770923e-01,7.27677882e-01],[-2.22614840e-01,-4.19386243e-03,1.79418653e-01]],[[-1.09661631e-02,9.07779276e-01,3.51931244e-01],[-3.10511500e-01,1.02411628e+00,-4.53750849e-01],[2.93800887e-02,-3.02088976e-01,-1.88015863e-01]],[[6.36432230e-01,-2.57279426e-01,3.32765222e-01],[5.06722368e-02,-3.51911038e-01,-1.33244604e-01],[-1.22238351e-02,1.88402146e-01,5.57176054e-01]],[[2.18276113e-01,-6.40700281e-01,2.95110017e-01],[-9.71053988e-02,-4.89372581e-01,-1.24467514e-01],[-7.09267914e-01,-6.36067867e-01,6.50438713e-03]],[[7.82683194e-01,-4.08650450e-02,9.15371478e-02],[-4.04971570e-01,-3.05938154e-01,1.12143487e-01],[3.03313345e-01,5.79345345e-01,1.26888007e-01]],[[6.41691014e-02,-1.32188767e-01,-2.15534791e-01],[4.06211585e-01,-1.61226794e-01,1.07659459e+00],[2.18390495e-01,3.56982619e-01,-6.30258024e-03]],[[2.26220652e-01,-5.79977334e-01,2.64148593e-01],[4.55151163e-02,-3.73189777e-01,3.49522650e-01],[-2.50938892e-01,-2.58265555e-01,4.51589823e-01]],[[-8.81664455e-02,-8.62694755e-02,1.64548010e-01],[3.36829931e-01,-5.56502938e-01,-5.22982955e-01],[2.92331517e-01,-2.92385012e-01,2.86138266e-01]],[[5.02225995e-01,3.61553639e-01,-2.79166162e-01],[1.53511390e-01,1.72868952e-01,1.43709689e-01],[-3.36270660e-01,-8.95633325e-02,-5.91573358e-01]],[[-6.41292408e-02,-6.20894551e-01,-2.88544130e-02],[-4.09678459e-01,-2.31491312e-01,-7.93244720e-01],[1.44306598e-02,1.90484062e-01,-4.64313179e-01]],[[-3.39570045e-01,7.19225854e-02,-1.55289793e+00],[-2.21507967e-01,1.13530025e-01,-8.23698401e-01],[-1.23939835e-01,-1.77022696e-01,2.52728164e-01]],[[1.09094135e-01,3.93911570e-01,2.28240654e-01],[-6.26770258e-01,-8.94513056e-02,1.73288748e-01],[3.01417112e-01,-5.28755724e-01,1.62452549e-01]],[[1.79363471e-02,-2.12270215e-01,-4.01016712e-01],[-4.03335631e-01,7.19934031e-02,1.87855825e-01],[5.19871600e-02,4.31589872e-01,-5.87835945e-02]],[[6.42632902e-01,-6.79706216e-01,-1.47286862e-01],[-1.97575152e-01,-4.81846094e-01,1.88663021e-01],[-2.93646634e-01,-6.21667385e-01,3.00114870e-01]],[[-5.23811504e-02,-2.98714787e-01,3.15991014e-01],[8.87937918e-02,3.90675783e-01,-6.02048635e-01],[-4.32126462e-01,-3.47754419e-01,-1.53715587e+00]],[[-5.96769989e-01,7.58686244e-01,-2.74335891e-01],[-3.00590217e-01,-3.37477803e-01,2.81197876e-01],[-2.83501297e-01,7.81861395e-02,-3.02803926e-02]],[[-2.52512276e-01,-6.54314756e-01,4.63408947e-01],[-1.53768778e-01,7.97583640e-01,2.84967333e-01],[9.03262794e-01,5.63651204e-01,3.57392490e-01]],[[1.96954086e-01,-2.09110200e-01,4.09183174e-01],[2.25945115e-02,-6.07125878e-01,-3.18198204e-01],[-3.64317000e-01,3.80225807e-01,-2.56164134e-01]],[[5.34513354e-01,7.43251890e-02,3.62656891e-01],[-9.95415822e-02,-6.44332990e-02,2.92151213e-01],[3.09146255e-01,1.63919717e-01,-3.39149773e-01]],[[3.65197062e-02,-2.19956055e-01,9.41927969e-01],[-7.33929515e-01,5.96045017e-01,-3.24508607e-01],[-1.89838335e-01,-2.86215216e-01,-2.17570454e-01]]],[[[0.65212727,0.4694944,-0.00143591],[-0.3046317,-0.31967068,0.32497635],[-0.44488424,-0.44499463,-0.32059032]],[[0.48603776,-0.28654146,-0.08574905],[0.3212185,0.14651194,-0.22674932],[0.31152436,-0.02828303,0.9788005]],[[0.5813412,-0.665465,0.2982055],[0.39125162,-0.5769469,-0.71945465],[0.47092664,-0.07883788,-0.24484459]],[[-0.05981657,-0.26652983,-0.4253628],[0.06804331,-0.15335262,0.66574454],[0.64335215,0.18067856,-0.40946013]],[[0.12379289,0.5243389,-0.23978199],[-0.7127108,0.12676556,-0.48761636],[0.30631202,0.6539698,0.02764462]],[[-0.07329668,-0.46350124,-0.3674244],[-0.37941003,0.4600012,0.49871844],[0.92672044,-0.08387876,0.24542925]],[[0.28170207,-0.31677318,-0.05342425],[0.9779334,-0.123744,0.34880525],[-0.01065159,0.19678813,-0.7515482]],[[0.04947037,-0.09750084,0.17219761],[-0.20591465,0.8109034,0.11958528],[-1.1731664,-0.1801927,0.5743978]],[[-0.92599756,0.0782599,-0.34993073],[0.5322765,0.30052954,-0.32083413],[0.04166908,-0.21791634,-0.00405647]],[[-0.41114464,-0.51221997,0.4025057],[0.36065277,0.03349198,0.5459231],[-0.4890186,0.05523025,-0.04867111]],[[-0.3871212,-0.38480482,0.56812763],[-0.26022762,-0.50975406,0.6280998],[-0.51022893,-0.38080466,0.39774612]],[[0.55978644,0.6692362,0.58922946],[-0.08261236,0.36306313,0.19508205],[0.28965074,0.8707954,0.2479191]],[[0.3352444,-0.26542574,0.92324346],[0.50119334,0.21945685,-0.42750332],[0.30529547,-0.04122313,0.56307226]],[[0.33512676,-0.60110724,0.5058015],[0.8950809,0.04776061,0.33367535],[0.61021304,0.03305222,0.6279928]],[[0.04657124,0.8707474,0.09647947],[0.49360883,-0.3132241,0.3432503],[0.33684283,-0.95422786,0.42120862]],[[-0.03087531,0.24811578,-0.2389035],[0.33554888,0.53352565,0.36114624],[0.23604728,0.04829286,0.8607659]],[[0.3012662,0.02956179,-0.33162823],[0.85809004,-0.19467884,0.19249967],[-0.53376114,-0.73494154,0.86915195]],[[0.21564633,0.42700544,-0.10052747],[-0.4204305,-0.31452686,-0.30353755],[-0.6746435,0.47127363,0.29547358]],[[-0.66389227,0.32488832,0.22662711],[0.24582863,0.48084295,-0.18594919],[0.33431125,-0.37424883,-0.0648045]],[[-0.30389023,-0.0408138,0.16612259],[0.3845934,-0.9030428,0.45521402],[0.4929644,-0.17804798,-0.47539002]],[[-0.08814684,-0.93691754,-0.82741034],[0.08324444,0.12421232,-0.05949071],[0.5044265,-0.50904655,0.23739596]],[[-0.4559288,1.2257355,-1.0404977],[0.3711097,0.8420455,-0.35184315],[1.1237214,0.30887067,-0.15428561]],[[-0.38634914,0.5789353,-0.07126185],[-0.05581987,0.17933072,-0.03081576],[0.6400953,0.76400715,-0.2030133]],[[0.1315662,0.06290992,0.8187937],[0.13152486,-0.36110696,-0.5519993],[-0.19155383,-0.45018566,1.4016525]],[[-0.34859636,0.7048214,-0.3792639],[-0.35727015,-0.70310795,0.09079561],[-0.5278164,0.7898586,0.114214]],[[-0.04325543,0.25415355,-0.57567686],[0.5688436,0.814641,0.63723695],[-0.61580354,-0.47066054,0.48876706]],[[-0.08491604,-0.22094733,0.2610129],[-0.2240558,0.06514584,0.05747626],[-0.4365931,0.03503829,0.32402208]],[[0.22453329,0.19379099,0.40038803],[-0.4365523,0.3213159,0.22367622],[0.57305986,-0.17413402,0.35876986]],[[0.7080424,0.39123723,-0.25613394],[-0.02819651,0.13792571,0.29233176],[0.11430458,0.46354967,0.32471788]],[[-0.1263861,0.139457,0.6140823],[0.01688259,0.19435544,-0.3153915],[-0.07512971,-0.08501067,-0.4421011]],[[0.2010106,-0.32480797,0.21765833],[-0.26633433,-0.48781484,-0.1010462],[-0.13796145,-0.9881592,0.19357163]],[[-0.02968544,-0.29939815,0.29259294],[1.3737693,-0.5563782,0.8232187],[-0.83699495,0.10864972,0.02810153]],[[-0.772323,-0.0098199,-0.04328625],[-0.00487111,-0.49726716,0.12025536],[0.07371859,-0.5480518,-0.13630873]],[[-0.05221975,0.12530987,0.6105973],[0.22191043,0.02778118,0.2997483],[-0.37297887,-0.27009496,0.49016342]],[[0.9386694,-0.34110656,0.13140975],[0.28270978,0.54201156,0.10336187],[0.440522,-0.22361739,-0.31060472]],[[-0.03150147,0.11823025,-0.21063983],[-0.19175585,-0.5927659,0.21575657],[-0.9430274,-0.06375071,-0.07261651]],[[0.76668066,-0.26482847,0.16655608],[0.3620694,0.03071688,0.25025746],[-0.14969862,-0.36876062,0.7277249]],[[0.30376896,-1.1269422,0.27813077],[-0.15189914,-0.1210373,0.81599927],[-0.00368598,0.2886451,0.80163354]],[[0.9182483,0.32566017,-0.39450917],[0.66777545,-0.337437,0.65338224],[0.8329001,0.21583046,-0.15359616]],[[-0.04941985,0.2405921,-0.22053541],[0.27698535,0.00461141,0.18400484],[-0.3418314,-0.16654699,0.30277693]]],[[[0.8518427,0.4553438,-0.17367993],[-0.22836155,0.09654216,-0.3677204],[-0.2713685,0.5668703,0.20651747]],[[-0.3595975,0.55671984,0.77595186],[-0.45322552,0.7775633,0.08287523],[-0.18783543,0.509683,-0.12312663]],[[-0.44175425,0.03949548,1.2338077],[-0.39267918,-0.27483666,-0.48812142],[-0.35706517,-0.31950247,0.090194]],[[-0.3230766,0.6748341,-1.0879198],[0.398452,0.2319682,-0.6573134],[-0.31128898,-0.3935054,0.34536347]],[[-0.59749216,0.33060724,-0.46019322],[-0.39895058,0.26761347,0.36525643],[-0.28935668,-0.61590123,-0.45032614]],[[0.27389696,0.21153487,-0.33810154],[-0.13900432,0.01595139,0.9504169],[-0.7402419,0.29582936,0.27047163]],[[0.03630626,-0.3212395,0.09585288],[0.14003159,-0.17834821,0.55339414],[-0.7567791,0.01895902,-0.4968064]],[[0.2585672,0.1421692,-0.37666807],[0.2847534,0.13083905,-0.31982917],[-0.10734227,0.10390101,-0.20452508]],[[-0.1767112,-0.32863635,0.45593026],[0.36290783,-0.08722538,-0.46587822],[0.22026893,0.14496031,0.1267396]],[[-0.30182812,-0.09944606,-0.37570956],[-0.98434216,-0.5047278,-0.1291487],[-0.03910059,0.11161005,0.17761008]],[[0.6330792,-0.45993236,-0.18553224],[-0.26190948,0.10855503,-0.23747383],[-0.22904047,0.02198514,-0.22292115]],[[-1.8244634,0.33354497,0.438226],[0.05598573,-0.29278278,0.5813892],[-0.34782708,0.836481,0.5779378]],[[0.42152953,0.24725047,0.7806491],[-0.7547206,0.42392004,-0.23264058],[0.52400005,0.5761845,-0.20376225]],[[-0.36155504,-0.34319058,-0.16953649],[-0.6744325,-0.08578202,-1.2333211],[-1.9951456,-0.3149066,-0.53021556]],[[0.04041855,0.10028828,0.4721928],[0.22745739,-0.1362803,-0.04154299],[0.9186581,-0.4241355,0.08128911]],[[0.72448146,0.11822815,-0.5887897],[-1.1272283,0.3025232,0.13671042],[-0.3434162,0.13287172,0.3754615]],[[0.08138408,0.6356217,-0.32067832],[0.30423516,0.3900873,-0.48945075],[0.24980702,0.03951237,0.17440964]],[[0.33435246,-0.19680883,-0.03091886],[-0.06379598,0.27486953,-0.03945317],[0.40754545,-0.01488367,0.32735458]],[[-0.47362584,-0.01316518,0.33774793],[-0.23929064,-0.13826078,-0.11371604],[-0.0688656,0.19470283,-0.47491333]],[[0.3197689,-0.2758979,-0.12592907],[0.69383687,-0.41017896,0.30247936],[0.27840674,-0.10946999,0.03794999]],[[-0.5972506,-0.46096385,0.7531016],[-0.09028643,-0.51511604,-0.4372588],[-0.03313118,-0.177909,-0.53754115]],[[1.4368433,-0.02717718,-0.7994601],[0.21908505,-0.22209026,0.02550784],[0.03094813,0.07357453,-0.27605987]],[[0.21425518,-0.15498221,0.7853061],[0.12741251,-0.13948677,0.46266213],[-0.57805145,-0.55730385,-0.04559946]],[[0.97153515,-0.83191675,-0.40146098],[0.36662003,0.24253333,-0.4132759],[-0.97248256,0.74144506,0.29525653]],[[0.3066001,-0.16551277,0.19212106],[0.12161745,0.59531575,-0.02010421],[0.6240678,-0.61207694,0.3075267]],[[0.6377184,-0.0137745,0.20765291],[-0.05986257,0.21932442,0.66309696],[-0.19432193,-1.0497614,0.49125296]],[[0.0711923,-0.25237885,-0.03063701],[0.27864337,0.71809375,0.02576452],[-0.36757258,0.05672487,-0.23748815]],[[0.38867125,0.38523236,0.35045347],[0.39112222,0.21066047,-0.4980275],[-0.6594681,0.16492204,0.67875504]],[[-0.38476768,-0.13811465,-0.744896],[0.28805044,0.6161154,-0.43807137],[0.15160184,0.04686327,-0.5185398]],[[-0.98260635,-0.46528694,-0.08192707],[0.41939944,-0.30914423,0.06358893],[-0.02021697,0.12402817,0.24782619]],[[-0.04894556,-0.30388188,-0.7819035],[-0.03385883,-0.28623596,-0.5000873],[0.5609739,-0.22553748,-0.4536365]],[[0.72232646,0.85838795,0.3822022],[-0.12890328,0.58042073,0.49172142],[0.06634958,0.18875206,0.14670797]],[[0.35846353,-0.7936418,-0.29626334],[0.933112,-0.65982074,0.5077416],[0.08340358,-0.9008119,0.36719373]],[[0.7649112,0.38138363,-0.08179182],[0.509958,0.04362039,-0.29773402],[0.37843484,-0.22303943,0.3855558]],[[0.02155452,0.37588415,-0.50559783],[-0.20905192,-0.3874134,0.1336366],[-0.46370286,0.11266916,0.2598501]],[[0.7842146,0.1617084,-0.04663647],[-0.10891385,-0.00306619,-1.6505661],[0.32934517,-0.07871222,-0.2733279]],[[-0.72720134,-0.0112147,-0.5902423],[0.5275015,-0.14289558,0.27996066],[-1.0803626,0.15045595,0.09521519]],[[0.46452105,0.04634522,-0.32499278],[-0.0194489,1.4462932,0.21160673],[-0.26067886,-0.22543852,-0.71091235]],[[0.14454895,-0.4866618,-0.43995592],[-0.2513689,-0.0250536,-0.21692382],[0.00279372,0.20083131,0.04663812]],[[-0.9314271,0.00525002,-0.5039786],[-0.07991727,-0.230342,-0.16560848],[-0.01603294,0.09899595,0.44529364]]],[[[-4.19480741e-01,-2.13276371e-02,2.12157462e-02],[8.71822000e-01,3.47263098e-01,2.26450153e-02],[7.69429982e-01,1.81301728e-01,4.16663438e-02]],[[4.04966801e-01,-6.28504992e-01,-3.45438153e-01],[7.06672907e-01,-2.24187031e-01,-7.32749933e-03],[-2.99500704e-01,1.26742810e-01,1.75928190e-01]],[[4.37589824e-01,-6.96964979e-01,4.17592674e-01],[4.34593201e-01,3.63747478e-01,4.27358150e-02],[4.17539388e-01,-4.85574365e-01,4.93774116e-01]],[[6.05682313e-01,4.54125375e-01,2.04478443e-01],[2.85774678e-01,4.57024276e-01,4.93220240e-03],[9.10718143e-02,4.65008378e-01,-1.15471229e-01]],[[1.77051798e-01,-3.67489517e-01,-5.10718107e-01],[6.29843652e-01,5.56419510e-03,1.01794980e-01],[-3.48372579e-01,7.40900487e-02,3.62338483e-01]],[[-2.10733131e-01,2.15619981e-01,-8.50304589e-02],[-4.41167116e-01,-2.26242915e-01,-1.64259627e-01],[2.03177482e-01,-2.32881755e-01,-4.46062058e-01]],[[-1.13881648e-01,5.06924391e-01,-7.17728972e-01],[2.25269079e-01,-1.14832200e-01,-7.36375749e-01],[4.55221504e-01,-2.22936705e-01,-7.76139051e-02]],[[-4.13045764e-01,-7.10353434e-01,3.15622121e-01],[8.98222253e-02,-3.97803038e-01,1.09460443e-01],[2.73695081e-01,3.41925919e-01,4.79289412e-01]],[[-2.26715282e-01,-7.54626840e-02,-7.94957995e-01],[-8.21392890e-03,-1.00199234e+00,9.20812309e-01],[-6.84169352e-01,-6.85984075e-01,3.14072013e-01]],[[4.22671288e-01,-3.80054146e-01,-1.60806030e-01],[-3.89427274e-01,-1.44954193e+00,-5.65225959e-01],[-1.15515220e+00,1.43271402e-01,1.08583915e+00]],[[5.47714114e-01,-1.29629508e-01,6.78230301e-02],[3.28957379e-01,1.11469471e+00,-1.99642688e-01],[-5.66556334e-01,-2.15516593e-02,-2.85815924e-01]],[[1.19565558e+00,-7.56499529e-01,-1.44483745e-01],[-1.03973615e+00,-5.92145398e-02,-5.97858205e-02],[2.88714260e-01,1.30236208e-01,-3.10668886e-01]],[[-1.32584408e-01,1.73271596e-01,-1.46469593e-01],[-1.91066697e-01,-1.34424061e-01,9.45167765e-02],[4.00878012e-01,-8.56555998e-01,1.31786406e-01]],[[-2.38150269e-01,2.01438546e-01,-5.04036725e-01],[2.91482598e-01,1.70054883e-01,3.57006460e-01],[4.34915632e-01,-3.03955406e-01,4.42930400e-01]],[[4.76173848e-01,2.19675139e-01,-8.07535499e-02],[4.13245037e-02,-2.14782566e-01,-2.80855626e-01],[-1.23320498e-01,4.64247823e-01,2.44415894e-01]],[[-7.37782359e-01,-1.41839907e-01,2.60283142e-01],[-1.19772649e+00,4.37048793e-01,-2.22006366e-01],[4.88565058e-01,-2.98251390e-01,7.12283969e-01]],[[-7.87503779e-01,1.44210696e-01,3.10797304e-01],[-1.65776402e-01,-2.88302571e-01,-8.47111419e-02],[-1.62503099e+00,6.02696300e-01,3.24844211e-01]],[[-8.41684461e-01,1.81650609e-01,2.26720020e-01],[4.95071024e-01,-1.96880639e-01,2.34915510e-01],[4.95574713e-01,9.33108747e-01,-2.07371444e-01]],[[7.05965936e-01,-9.16191116e-02,-1.62373260e-01],[2.42062524e-01,1.33112326e-01,-8.32442284e-01],[-3.80560160e-01,1.85309500e-01,1.77339256e-01]],[[-3.08065355e-01,1.13035545e-01,3.58313531e-01],[-2.60411482e-03,-8.53113174e-01,-3.33278984e-01],[-8.56366009e-02,5.20887136e-01,-3.64110559e-01]],[[3.34265769e-01,5.46665080e-02,-5.73926210e-01],[2.65114278e-01,-5.08349359e-01,5.58293331e-03],[-4.21023905e-01,-6.27791211e-02,3.05376351e-01]],[[-6.05234914e-02,5.92144370e-01,7.04268396e-01],[4.51479703e-01,-6.00162983e-01,3.75660598e-01],[-8.22453380e-01,-8.66937339e-02,-6.11919761e-01]],[[1.00229234e-02,1.67280361e-01,-1.94227040e-01],[9.30040896e-01,2.06263453e-01,-5.16375303e-01],[-1.55726016e-01,-4.21691656e-01,-7.36382008e-01]],[[-2.20277011e-01,4.09629256e-01,-1.03712551e-01],[-5.52624054e-02,-2.25546275e-04,-2.14003369e-01],[5.78585505e-01,4.59233820e-02,-5.50903499e-01]],[[-1.80695087e-01,-6.98578477e-01,-9.05758888e-02],[-5.29511012e-02,3.17588001e-02,1.31698608e-01],[-7.75483489e-01,-2.22875640e-01,-3.71350914e-01]],[[3.43035251e-01,-4.34362352e-01,3.08147043e-01],[-7.92112648e-01,1.16672739e-02,2.87949890e-01],[-7.52555430e-01,8.28448772e-01,6.66643381e-01]],[[-3.60793434e-02,9.01595429e-02,-3.00338715e-01],[-3.84306535e-02,1.90026499e-02,-6.82995856e-01],[3.38820130e-01,-9.36176360e-01,-1.06769717e+00]],[[-1.09408863e-01,-9.39406306e-02,5.24576664e-01],[-9.16386321e-02,-7.52675951e-01,-1.16318427e-01],[4.29680705e-01,1.74889430e-01,4.78806615e-01]],[[2.30430990e-01,2.52299219e-01,2.20158160e-01],[-6.73447847e-02,2.96362996e-01,-2.68616199e-01],[-8.06158900e-01,5.52543759e-01,-4.37712848e-01]],[[-6.28883660e-01,1.00111835e-01,-5.29677272e-01],[-1.58200964e-01,2.56403446e-01,-2.52358764e-01],[-1.99350238e-01,-7.13740110e-01,3.90217751e-01]],[[-2.25204960e-01,-4.48219419e-01,4.12533313e-01],[4.59716529e-01,-5.22405326e-01,2.64799565e-01],[-1.80211365e-01,-4.73304540e-01,4.03611474e-02]],[[1.26540273e-01,-5.52587092e-01,-3.00415456e-01],[-2.41233692e-01,1.68342888e-01,-3.04857194e-01],[-8.18227708e-01,-4.18436527e-01,-3.82324047e-02]],[[4.75067198e-01,9.24258605e-02,-4.50067550e-01],[-4.40887690e-01,-2.63818622e-01,2.38600731e-01],[-1.96882412e-01,8.07146311e-01,-5.12206972e-01]],[[-4.53223996e-02,1.38861602e-02,-1.32645234e-01],[1.38100505e-01,-1.30183510e-02,-2.15135396e-01],[1.66656673e-02,-4.06963751e-02,3.41016680e-01]],[[2.83321142e-01,-7.55417943e-01,9.93050456e-01],[2.38728493e-01,-6.33555800e-02,4.36893106e-03],[-3.74844909e-01,-7.48461932e-02,-2.94592857e-01]],[[-3.48127514e-01,4.44369823e-01,-3.34367067e-01],[2.64317930e-01,3.71636264e-03,-4.86792922e-01],[-2.95560390e-01,5.01999378e-01,-3.16065215e-02]],[[-5.05165696e-01,3.21344227e-01,1.01284087e+00],[-6.77546382e-01,3.32640916e-01,-6.74481764e-02],[4.10372764e-02,-5.27072884e-02,1.14108336e+00]],[[2.66651273e-01,-3.23229760e-01,5.69642372e-02],[-1.03454936e+00,-5.71896195e-01,-4.55633521e-01],[-6.46434963e-01,-1.74453035e-01,-5.85099995e-01]],[[1.05353940e+00,-3.05781573e-01,-3.07011873e-01],[3.37329596e-01,3.99144500e-01,1.64322048e-01],[2.51297027e-01,3.80335152e-01,-6.47194147e-01]],[[-3.88556093e-01,4.76918817e-01,-9.53397676e-02],[1.50984809e-01,-3.70754331e-01,-6.12609863e-01],[-5.02870858e-01,-7.26533085e-02,-1.34644983e-03]]],[[[-8.09231758e-01,4.96640712e-01,-6.56436265e-01],[-2.07748562e-01,7.09853470e-01,2.89158136e-01],[1.21795118e-01,1.17338710e-01,1.33897662e-01]],[[-6.45896196e-01,2.52441406e-01,-3.45339686e-01],[2.10717544e-01,5.28576910e-01,-3.75157863e-01],[-4.23908651e-01,1.28359601e-01,2.13160649e-01]],[[-9.06713754e-02,4.57231514e-02,1.97348464e+00],[-1.48383703e-03,5.30876629e-02,2.03231499e-01],[-4.06347573e-01,1.19811617e-01,-1.12777077e-01]],[[5.67769885e-01,1.43627554e-01,3.26586545e-01],[-3.33965629e-01,1.03533007e-01,-2.40820006e-01],[7.68192112e-01,1.29367877e-02,-5.03933847e-01]],[[2.30482042e-01,2.89829195e-01,-5.32680571e-01],[3.20857197e-01,8.48436132e-02,4.90492404e-01],[1.13265085e+00,2.50149935e-01,1.93670884e-01]],[[-5.82294583e-01,-9.36029777e-02,9.12561119e-01],[1.15662366e-01,1.68495864e-01,6.53412879e-01],[-1.53548911e-01,1.99049175e-01,6.58776820e-01]],[[-2.10986167e-01,-4.73358810e-01,-9.58806634e-01],[-4.37716581e-02,2.12010920e-01,-8.00403476e-01],[8.25770617e-01,3.96110803e-01,-2.56660789e-01]],[[-4.40278649e-01,-6.37205243e-01,1.72436059e-01],[2.72209674e-01,6.43721104e-01,-2.14947730e-01],[5.57976246e-01,-7.12848723e-01,1.60801992e-01]],[[1.45184845e-01,-4.97488737e-01,-1.12225495e-01],[-5.16462564e-01,1.10974893e-01,-5.99650741e-02],[-6.44026101e-02,6.14368320e-01,7.78955877e-01]],[[-1.23394363e-01,-9.62901294e-01,1.94258280e-02],[-3.86811942e-01,-4.06843245e-01,4.67779078e-02],[-1.06921864e+00,1.90294713e-01,4.84459609e-01]],[[1.58636853e-01,7.44256020e-01,-2.95791268e-01],[1.65223926e-01,4.72578794e-01,-1.83943003e-01],[8.03430617e-01,3.34789455e-02,-1.02491760e+00]],[[-2.61641353e-01,4.93758023e-01,2.24119406e-02],[-6.34103358e-01,6.57852232e-01,7.39979625e-01],[-8.66228640e-01,-1.14798293e-01,4.35777158e-01]],[[7.70689309e-01,-5.14330208e-01,1.57073349e-01],[8.07312727e-01,-1.00385928e+00,3.52071732e-01],[1.18748553e-01,-8.97058189e-01,2.55865961e-01]],[[-2.79173970e-01,-2.52185464e-01,3.74331802e-01],[2.11710021e-01,2.50436902e-01,-2.79135734e-01],[-2.95226246e-01,1.26490757e-01,-5.45552254e-01]],[[5.01776397e-01,-8.95187929e-02,-5.32469869e-01],[5.17596975e-02,2.57079095e-01,-4.09669250e-01],[2.98286945e-01,1.70785382e-01,-5.83812371e-02]],[[2.80712366e-01,-1.27398269e-02,-4.54067960e-02],[-3.73126686e-01,-1.97878420e-01,-1.76524282e-01],[-9.18679684e-02,6.43480062e-01,1.38147938e+00]],[[1.63873211e-02,-6.96546361e-02,-2.81168401e-01],[-1.70745388e-01,-6.42862678e-01,-1.85049996e-01],[-4.31463748e-01,2.15963304e-01,-5.64097762e-01]],[[-2.22078830e-01,-4.84769940e-01,-5.23290217e-01],[-1.31766111e-01,7.67304897e-01,1.77449808e-01],[3.57477605e-01,6.06910996e-02,-3.29950750e-01]],[[-4.17682916e-01,1.67991683e-01,1.51321337e-01],[2.87148744e-01,-4.48501199e-01,1.11390519e+00],[-4.46160555e-01,7.63736427e-01,-3.82815778e-01]],[[-4.23236787e-02,-6.72704950e-02,1.03121974e-01],[3.93173665e-01,-8.68538395e-02,-3.06669414e-01],[1.40845194e-01,-4.18881357e-01,-7.06543922e-01]],[[4.34212089e-02,-3.16923767e-01,1.15675426e+00],[-8.71658742e-01,-5.93915224e-01,3.36608738e-01],[-6.12126410e-01,-3.01444709e-01,-4.33953464e-01]],[[2.57358551e-01,3.08755022e-02,1.68021783e-01],[-4.95009944e-02,-5.35987794e-01,4.55292165e-01],[1.09530382e-01,3.36502306e-02,3.53347927e-01]],[[6.32080287e-02,3.32685500e-01,-2.46762291e-01],[3.75578493e-01,4.02846374e-02,3.27559143e-01],[-7.30260611e-02,-1.10473588e-01,3.39620382e-01]],[[-1.48278445e-01,-3.81923914e-01,-4.03270483e-01],[-4.91312593e-01,6.74744919e-02,-6.70468748e-01],[2.28078172e-01,7.83113360e-01,-1.44032821e-01]],[[4.48339641e-01,-3.27215642e-01,-4.94244814e-01],[-2.03571934e-02,-1.23527639e-01,5.09331703e-01],[3.75920795e-02,-2.13594791e-02,-4.02797610e-01]],[[1.16914898e-01,-1.40533492e-01,-8.38992074e-02],[-9.65029180e-01,-2.27922603e-01,1.99393108e-01],[-1.10629886e-01,-2.47715622e-01,3.80284548e-01]],[[5.26679575e-01,4.07141715e-01,-3.19346368e-01],[7.88803220e-01,-4.88861233e-01,-1.42049715e-01],[6.02599621e-01,5.42295754e-01,1.20319584e-02]],[[9.07000080e-02,-2.47843191e-01,-3.17349166e-01],[-2.51451492e-01,-1.56670287e-01,-2.78782010e-01],[-4.21324968e-01,5.78479350e-01,4.80935305e-01]],[[-2.44947731e-01,2.56747097e-01,5.96412361e-01],[-4.03156996e-01,-7.98864484e-01,1.28357381e-01],[6.19487524e-01,3.40181351e-01,7.68868625e-02]],[[-9.94312763e-01,-1.03454515e-01,4.46373135e-01],[3.15810204e-01,-3.49469244e-01,3.07520390e-01],[1.00440048e-01,-8.98651034e-02,9.20040309e-02]],[[-4.87953052e-03,-2.71279722e-01,-4.95808780e-01],[2.95828372e-01,-4.02847737e-01,-9.53206062e-01],[-2.51223743e-01,-2.09879473e-01,-6.10599160e-01]],[[8.86798441e-01,1.28393137e+00,-1.52784571e-01],[-4.65902925e-01,6.31965756e-01,-6.14517987e-01],[-9.97822940e-01,9.31809306e-01,-1.31645846e+00]],[[-2.37827212e-01,2.21030980e-01,1.42555907e-01],[6.95944652e-02,1.77506953e-02,3.29997331e-01],[-1.10617340e+00,2.67947823e-01,3.04316193e-01]],[[2.04959914e-01,-8.41844320e-01,-1.62348166e-01],[1.97637260e-01,-1.89544901e-01,4.63648558e-01],[-1.92644796e-03,-6.85195848e-02,7.45896697e-01]],[[-3.31053406e-01,-1.95452403e-02,-4.03025337e-02],[-5.35476327e-01,7.32002974e-01,2.76699275e-01],[-5.09548247e-01,-5.52048087e-01,5.77615261e-01]],[[2.26463135e-02,3.74420494e-01,4.75644797e-01],[3.58047038e-01,-4.84128624e-01,-7.51133859e-01],[-5.07576056e-02,1.09024934e-01,-6.72530711e-01]],[[-7.50365436e-01,2.36117810e-01,3.61048579e-01],[1.63818881e-01,1.24265164e-01,-6.44444346e-01],[-1.24384248e+00,1.74037710e-01,1.07407719e-01]],[[8.46615434e-02,1.50249183e-01,-1.97040290e-02],[8.03733587e-01,-2.29807690e-01,2.23731086e-01],[-4.80223119e-01,6.25354350e-02,5.71564615e-01]],[[-4.66098398e-01,-2.06278056e-01,-3.18623811e-01],[1.04687802e-01,-2.32568517e-01,-1.47910789e-01],[-4.52202052e-01,4.58675146e-01,-3.69371139e-02]],[[-8.05933118e-01,3.89519818e-02,-6.33263364e-02],[4.00577672e-02,-2.18536347e-01,3.21644515e-01],[-3.06740999e-01,-3.90379429e-01,2.68474430e-01]]],[[[0.3438862,-0.95723915,-0.18620099],[0.11802025,0.37262464,0.00351478],[0.37246415,0.02137979,-0.4184319]],[[-0.1511717,0.23596543,0.47670853],[0.34069294,-0.6355124,-0.27548102],[0.7007627,-0.20932776,0.5749333]],[[-0.12908962,0.29004228,-0.08436327],[-0.12888469,-0.33272114,0.9518334],[0.5201973,-0.31770673,0.01812777]],[[-0.25409055,0.34220272,-0.26396358],[-0.75672626,-1.0737184,0.0275276],[-0.6554715,-0.30035686,0.72147685]],[[0.9250085,0.19117211,0.05890776],[-0.3463931,-0.36103597,-0.03345644],[-0.19910043,0.05306446,0.15427569]],[[0.10829654,-0.15286176,0.13254741],[-0.24251652,-0.4870923,-0.49381235],[0.301556,-0.3588857,0.25312695]],[[0.75952685,-0.32495838,-0.5553412],[0.6868102,-0.5318377,0.14332184],[0.3897897,-0.28719333,-0.220657]],[[-0.01415026,-1.3616408,-0.6093979],[0.0896653,0.6030723,0.9315515],[0.4351801,-0.37743443,-0.623772]],[[-0.09726746,0.03124877,0.29055625],[-0.07307898,-1.1272427,0.66592014],[-0.88754755,-0.3864709,-0.7312702]],[[-0.7071345,0.1928216,-0.6996067],[-0.9628497,-0.1340393,-0.39390764],[-0.49741605,-0.18603994,-1.2162637]],[[-0.22681323,-0.40615377,-0.37024912],[0.95386875,-0.29447997,-0.4314198],[0.5493353,0.04455584,-0.16086698]],[[0.11761826,-0.19627233,-0.00838781],[0.06723563,0.39141077,0.35798255],[-0.28080127,-0.45418018,-0.07451046]],[[0.04356082,0.28664047,-0.12119646],[0.58306473,-0.284844,-0.37801138],[0.72555006,0.76255774,-0.5346577]],[[0.37812585,-0.14833061,0.09690873],[-0.65585226,-0.05156358,0.23693545],[0.05692861,-0.27588034,0.27539864]],[[0.4637088,0.06339964,-0.86583203],[0.87098366,0.5946488,-0.58215094],[0.31574774,0.39996776,0.62110573]],[[-0.06024456,-0.25686517,-0.44510618],[-0.9430742,-0.03646918,0.50009364],[-0.39587685,-0.3450505,0.19929479]],[[-0.14017032,0.27436712,0.08077385],[-0.0795901,0.07291868,-0.6043376],[-0.72308415,-0.11258119,-0.05366051]],[[-0.7258526,0.2564993,0.12804468],[-0.1858322,0.22355731,0.48417628],[0.9508309,0.64787894,0.19314267]],[[-0.27854708,0.46366015,-0.02478888],[0.07432187,-0.5933118,0.6062957],[0.42555606,-0.5636279,0.07361852]],[[-0.16062973,0.2655399,-0.6062383],[0.3730022,-0.00320874,-0.80639726],[-0.20773079,-0.15094003,0.28715906]],[[-0.15737583,0.17959192,-0.33635822],[-0.4336999,0.6833232,-0.05181418],[-0.52671516,0.26199427,-0.37647206]],[[0.25995564,-0.12217391,-0.04142507],[-0.0483225,-0.51207453,0.49298885],[-0.43375275,0.0361288,0.3000294]],[[-0.2604379,0.14392471,0.32717648],[-0.02939549,0.24802993,-0.72614825],[0.5675266,0.7068572,-0.01997902]],[[0.54988194,-0.02990893,-0.79671973],[-0.3883554,-0.80796766,-0.12065699],[-0.09480834,-0.28088328,0.47493625]],[[-0.54764986,-0.11476407,-0.03071789],[-0.63726026,0.61848855,-0.5026423],[-0.66925836,0.02326841,0.45797402]],[[-0.33483845,0.794167,-0.06064751],[0.18040703,-0.39335573,0.67869693],[0.27213764,0.73566335,0.5010376]],[[0.31929842,0.41121054,0.14625804],[0.657953,0.04659261,0.09865398],[0.8033003,0.22973943,-0.09229098]],[[0.13841066,-0.39877722,0.02471378],[-0.7254248,0.67700416,0.0881942],[-0.3790642,0.04425754,-0.588253]],[[0.02527791,0.11911475,0.7801234],[0.49719378,0.40488958,-0.5886114],[-0.5385754,0.53350997,0.3644623]],[[0.07609104,0.13318975,-0.05126562],[-0.28706798,-0.80814433,0.6963193],[0.60916865,0.35471636,0.00492654]],[[0.02669054,-0.38105196,-0.4533745],[-0.37020436,-0.6695524,-0.14020821],[0.21156278,-0.32480475,0.28072423]],[[-0.5001873,0.01851625,0.6904836],[0.19281802,0.02426833,0.925269],[-0.01502641,0.28323892,0.427155]],[[-0.3865288,-0.5162991,-0.08731884],[-0.6061333,0.38252065,-0.40876815],[0.02767959,-0.40163368,0.25159582]],[[-0.64187014,0.12558815,-0.24882641],[-0.29294452,0.06935333,0.24430926],[0.01607881,0.8098353,0.28205183]],[[0.19866769,-0.34112662,-0.06500801],[-0.5860521,0.42427588,0.02022363],[-0.20014416,0.01264375,0.76531315]],[[-0.13271387,0.42154866,-0.0868546],[-0.2568617,0.30482426,-0.07611986],[0.7859249,-0.22887221,-0.23597229]],[[0.17701934,0.39870492,0.37157306],[0.854349,-0.05389044,0.14237373],[0.5955564,0.42668125,0.60949665]],[[0.38980412,-0.91374284,0.89775985],[-0.64482516,0.8758851,0.29127535],[-0.29532945,0.30038428,0.6731058]],[[0.6907558,0.09947792,0.02296241],[0.2800149,-1.1564916,-0.2685931],[0.15163581,-0.2572288,0.5180125]],[[-0.40629032,-0.5434873,0.65594757],[-0.20703045,-0.6522738,0.33855408],[0.05979077,-0.4076086,-0.34620488]]],[[[1.32513821e-01,2.02113822e-01,8.26698318e-02],[5.25848567e-01,-4.49031889e-01,-7.26373851e-01],[-2.14705512e-01,1.25049546e-01,1.00715563e-01]],[[3.67958874e-01,-2.49765083e-01,4.16123345e-02],[2.39082918e-01,-8.68455917e-02,-7.41414785e-01],[1.14509857e+00,-4.34354335e-01,-1.61725819e-01]],[[-1.49545953e-01,-1.32129133e-01,-4.39707905e-01],[4.88137603e-02,2.55694211e-01,-7.58096650e-02],[-2.12403670e-01,-1.47923157e-01,2.94262588e-01]],[[-1.13193668e-01,-5.97262561e-01,-7.20449984e-01],[3.00326526e-01,-1.45529166e-01,-1.08229425e-02],[-1.14046371e+00,-5.42312920e-01,-5.37224948e-01]],[[-2.57480424e-02,5.03196239e-01,8.56514722e-02],[2.52449811e-01,-3.23501080e-01,5.04202306e-01],[-3.99962604e-01,-1.36073291e-01,-4.58265960e-01]],[[-3.23514864e-02,-2.27927268e-01,8.28732252e-01],[-9.64041412e-01,-5.27506649e-01,2.05679640e-01],[5.04186936e-02,2.25421488e-02,3.65679830e-01]],[[5.77714026e-01,1.77428350e-01,5.52360356e-01],[9.60598767e-01,-3.27553958e-01,3.72420728e-01],[2.03213528e-01,-7.00818300e-02,-8.67221132e-02]],[[1.05265009e+00,-3.22591722e-01,2.05891058e-01],[4.08442467e-01,-4.42907423e-01,4.39788342e-01],[-9.37929824e-02,-6.63663372e-02,4.47963953e-01]],[[1.60419822e-01,-1.87162742e-01,1.98907092e-01],[-3.49468812e-02,-8.24612528e-02,-7.46620297e-01],[3.50552946e-01,-2.93281823e-01,-3.47960919e-01]],[[5.00441790e-01,3.83439183e-01,-1.96045414e-01],[2.48076826e-01,4.26854283e-01,-3.56984466e-01],[8.82558227e-01,-1.16071381e-01,3.41234267e-01]],[[5.60328066e-01,-4.07418758e-01,-1.27418146e-01],[4.19663548e-01,-1.62224874e-01,7.94261098e-01],[-1.48811668e-01,1.57126069e-01,4.94760752e-01]],[[-6.18914545e-01,6.13364987e-02,-3.75412583e-01],[-5.57593465e-01,-5.38934827e-01,6.11971140e-01],[-2.52797931e-01,-1.82304516e-01,6.22665919e-02]],[[-8.73275697e-02,7.05067635e-01,5.18234611e-01],[-8.74278247e-01,8.64610523e-02,-4.61324185e-01],[-1.31407559e-01,-6.01348281e-01,-2.62894556e-02]],[[-6.81904614e-01,-9.80258062e-02,-5.82278408e-02],[2.85466969e-01,1.86338156e-01,-2.24107787e-01],[-5.53216934e-01,-5.94307892e-02,2.04459429e-01]],[[-4.88221906e-02,-1.89067468e-01,-5.37278771e-01],[-6.24914169e-01,3.08765560e-01,-1.54411644e-01],[-2.42290702e-02,-3.39521021e-02,-4.81687993e-01]],[[-7.94148624e-01,-5.66605091e-01,-8.03334296e-01],[1.84643522e-01,1.74061909e-01,3.31172496e-01],[-4.18771617e-02,4.09381866e-01,-1.17047036e+00]],[[2.38127440e-01,4.81188953e-01,3.72669905e-01],[-1.29514262e-01,-1.66882396e-01,-4.65851694e-01],[-9.17387530e-02,2.42825598e-01,4.62623946e-02]],[[-8.71417761e-01,4.31017041e-01,-2.98149377e-01],[6.89972239e-03,2.30491206e-01,9.34974104e-02],[2.68292159e-01,-7.30295539e-01,2.89405227e-01]],[[3.88511926e-01,-9.38790262e-01,-6.78363144e-01],[7.47290999e-02,-2.75486350e-01,3.95659834e-01],[3.45183134e-01,-2.64422446e-01,-1.77731067e-01]],[[-4.94626731e-01,-2.20498234e-01,8.20580959e-01],[-1.26881570e-01,7.53506184e-01,1.84531316e-01],[-8.51503834e-02,-7.27887675e-02,1.18969417e+00]],[[-3.46029550e-01,-2.75812119e-01,-6.99996591e-01],[-4.34408039e-01,-8.48717242e-02,4.76004519e-02],[8.00298035e-01,2.84839809e-01,-3.17050576e-01]],[[-2.26119667e-01,5.45902371e-01,-8.00383270e-01],[1.79349124e-01,-5.35868183e-02,2.15645313e-01],[-4.06827331e-01,-2.09789962e-01,2.82533258e-01]],[[1.82567686e-02,5.08924313e-02,4.98380601e-01],[-5.50453246e-01,1.22307450e-01,-8.46111700e-02],[4.40516174e-02,-4.12458442e-02,2.20965102e-01]],[[3.95379066e-01,-4.57046270e-01,3.52390230e-01],[1.73621297e-01,-3.59934807e-01,3.94611746e-01],[4.63625312e-01,8.48078877e-02,1.25077236e+00]],[[6.08020425e-01,1.00319907e-01,1.09929621e+00],[-6.55189753e-02,2.66837686e-01,-1.44575194e-01],[8.44489574e-01,2.21626937e-01,4.14999336e-01]],[[-4.15240675e-01,-2.71566361e-01,-4.98053461e-01],[-2.66959935e-01,3.87113154e-01,4.65746999e-01],[1.41742274e-01,4.27302122e-01,-7.97872424e-01]],[[-4.52194542e-01,-3.89090598e-01,4.95295018e-01],[-5.45035243e-01,9.07510370e-02,6.14676595e-01],[-3.42360623e-02,-6.42523646e-01,9.92426276e-01]],[[-2.18048349e-01,-1.20547213e-01,2.36127868e-01],[4.41019446e-01,-6.51892722e-01,5.27248204e-01],[1.74583769e+00,2.10751012e-01,4.02684897e-01]],[[-8.73802900e-02,-1.70236956e-02,-3.41214016e-02],[3.62811476e-01,-3.27733010e-01,-2.60190398e-01],[1.22549105e+00,-3.65715250e-02,-8.88852775e-02]],[[-5.20070136e-01,1.14720702e-01,3.86093765e-01],[-4.96097952e-01,-8.84114355e-02,4.51992959e-01],[3.14119637e-01,3.05288136e-01,6.02612376e-01]],[[-1.16029531e-01,-1.07564174e-01,1.08825028e-01],[3.14808369e-01,-2.57248163e-01,-2.53500253e-01],[7.98981190e-01,-4.87228096e-01,3.69970128e-02]],[[2.54844457e-01,1.53072581e-01,4.14049298e-01],[-7.04974294e-01,3.44040275e-01,1.98211018e-02],[-1.51293471e-01,4.89864469e-01,2.44147003e-01]],[[1.11945295e+00,-5.48882365e-01,3.19761723e-01],[9.31698382e-01,-3.36168766e-01,7.11127698e-01],[-1.37281604e-04,-1.36519313e-01,4.83097732e-01]],[[4.26646531e-01,1.02119893e-01,3.84705931e-01],[1.98605582e-01,5.56082726e-01,-1.45433724e-01],[-3.84114534e-01,-3.94483835e-01,2.42586911e-01]],[[2.66046286e-01,8.14470798e-02,2.64664203e-01],[-1.53729826e-01,2.79566020e-01,1.65578038e-01],[4.49777246e-01,-8.18215787e-01,-1.51411340e-01]],[[-5.04152775e-01,2.83231467e-01,4.02509421e-01],[-8.64933506e-02,-7.74574757e-01,-8.67655277e-02],[-7.24407984e-03,-6.23668246e-02,6.68992639e-01]],[[5.63980937e-01,-4.76616621e-03,-5.05879879e-01],[-8.25817049e-01,-6.39822900e-01,2.03450933e-01],[-1.82866752e-01,8.65692273e-02,6.41697526e-01]],[[4.52040255e-01,4.69203740e-01,-3.25422227e-01],[6.42597556e-01,5.89690089e-01,-9.00054336e-01],[9.14049923e-01,2.08142221e-01,9.79114056e-01]],[[-4.34789538e-01,3.59881163e-01,8.29910815e-01],[-4.34685409e-01,-2.08365284e-02,2.48722374e-01],[-9.47637036e-02,-2.68226564e-01,-4.49712634e-01]],[[2.00164616e-01,3.48244399e-01,5.37847243e-02],[-1.67565435e-01,2.36269787e-01,-4.18078244e-01],[1.12087101e-01,7.98254609e-02,-3.14786583e-01]]],[[[-2.90140271e-01,-5.00812769e-01,5.27977467e-01],[4.87445563e-01,1.91774383e-01,1.42176570e-02],[-5.58133721e-01,-3.67442161e-01,2.47422624e-02]],[[9.26712334e-01,-5.93551807e-03,-1.05816650e+00],[-3.83475184e-01,1.21521190e-01,-1.37641680e+00],[2.68626601e-01,5.32874942e-01,1.66463479e-03]],[[-1.92422897e-01,4.13754791e-01,9.73520130e-02],[2.05177456e-01,-6.38510227e-01,2.63170063e-01],[-1.53220668e-02,-9.45303366e-02,-3.61463964e-01]],[[-4.96128798e-01,-4.20999408e-01,-3.87811452e-01],[-4.89818692e-01,-3.12212463e-02,-1.59627929e-01],[6.46333396e-01,9.40275118e-02,3.12653542e-01]],[[-5.17066538e-01,4.06837389e-02,1.66858554e-01],[1.76906303e-01,-3.41274053e-01,2.12301776e-01],[-1.69857547e-01,-5.42910099e-01,1.65001512e-01]],[[4.63062346e-01,-3.82277407e-02,-6.71343505e-01],[-3.70076060e-01,-3.62538815e-01,-1.17675352e+00],[4.67313766e-01,-9.29984570e-01,-6.41795322e-02]],[[7.36616910e-01,2.18335241e-01,2.29954138e-01],[-3.26178998e-01,2.12838158e-01,7.36261904e-02],[-5.19972563e-01,3.43553536e-02,-1.46716729e-01]],[[8.09177935e-01,2.10301995e-01,-8.55508268e-01],[-3.35093498e-01,1.22032130e+00,-5.71880996e-01],[1.13449514e-01,1.97487295e-01,5.52614570e-01]],[[-3.68701369e-01,4.67046440e-01,-5.37960708e-01],[1.55184388e-01,-1.43460408e-01,5.36419041e-02],[7.56493568e-01,1.94287494e-01,1.37357444e-01]],[[1.26179051e+00,2.28433102e-01,-5.89815676e-01],[2.37841308e-01,-6.34126186e-01,-5.89857638e-01],[4.16439056e-01,3.83069634e-01,-3.78373861e-01]],[[9.49662179e-02,-2.41465732e-01,3.68924201e-01],[-6.44093931e-01,2.46522343e-03,5.89377344e-01],[-1.53287470e-01,-3.82896304e-01,1.15285039e+00]],[[-5.19062221e-01,-4.51815665e-01,-6.51493013e-01],[2.24809155e-01,3.06306947e-02,1.53101087e-01],[9.57023874e-02,-4.61897552e-01,1.31783530e-01]],[[3.42719764e-01,2.50171572e-01,9.40459967e-02],[-1.84713125e-01,-4.70982939e-02,-6.54096842e-01],[-7.84495234e-01,2.62619853e-01,-6.89812839e-01]],[[6.07615374e-02,3.06167275e-01,7.88048357e-02],[-1.61281928e-01,-2.26752713e-01,-8.95867348e-01],[-3.96280587e-01,-1.81820076e-02,1.14011824e+00]],[[-1.60662442e-01,-2.63496369e-01,-3.09696048e-01],[-2.44076878e-01,-1.99530959e-01,-1.13784575e+00],[-8.57679248e-02,1.98784202e-01,5.13704836e-01]],[[1.13042146e-01,-3.76760215e-01,9.41938832e-02],[-1.04986824e-01,1.54180020e-01,-3.13877016e-01],[1.62281907e+00,-3.04001033e-01,-2.03168601e-01]],[[-7.69284666e-02,8.36883664e-01,2.12984368e-01],[9.49640751e-01,-7.24922001e-01,-6.88683271e-01],[-3.51950437e-01,1.49058579e-02,9.27199125e-02]],[[-5.47050953e-01,-4.07109782e-02,-4.60150510e-01],[5.79762757e-01,6.42324518e-03,-8.85955453e-01],[-3.29954594e-01,-3.11375797e-01,-1.83697268e-01]],[[2.81304820e-03,5.30743524e-02,-1.24561206e-01],[4.94507812e-02,3.48583221e-01,2.62377888e-01],[6.84473515e-01,-1.69254273e-01,-5.77140808e-01]],[[3.85251433e-01,-7.19447061e-02,5.50383210e-01],[-1.86686337e-01,-6.44553602e-01,-1.71574265e-01],[-3.78308833e-01,3.38638544e-01,-1.01512802e+00]],[[5.13961077e-01,3.12985443e-02,4.24815454e-02],[1.28680304e-01,-2.99643219e-01,2.18162239e-02],[1.11064978e-01,-3.38679731e-01,3.54555309e-01]],[[2.98986614e-01,-5.63634038e-01,1.02933608e-01],[-2.80490100e-01,-3.01614255e-01,4.84869406e-02],[2.71633923e-01,-2.59970650e-02,-1.79016277e-01]],[[8.14895034e-02,5.53326070e-01,-1.78288326e-01],[8.21927264e-02,6.22821271e-01,-5.85574210e-01],[-1.76644534e-01,-2.05195412e-01,-9.99112725e-02]],[[5.38928032e-01,2.92060822e-01,1.17661282e-01],[-4.17236537e-01,-2.38475800e-01,1.03880055e-01],[-2.53366530e-01,-2.66642749e-01,-7.99622238e-01]],[[-6.17498867e-02,9.36461762e-02,2.44705647e-01],[7.09996462e-01,1.79272890e-01,4.93922025e-01],[-8.10960948e-01,-1.48040071e-01,5.34034252e-01]],[[2.81111952e-02,9.88215208e-02,-2.26985794e-02],[-9.04617608e-01,-1.53421789e-01,-5.23662508e-01],[1.24507099e-01,1.74563557e-01,2.59837024e-02]],[[3.09588313e-01,3.02150369e-01,-4.68248188e-01],[-2.11334094e-01,2.11938262e-01,-4.24706861e-02],[1.30880609e-01,-1.61841199e-01,-7.27458969e-02]],[[9.82860848e-02,3.88480648e-02,7.03851059e-02],[-1.72363415e-01,1.75265193e-01,-1.56194389e-01],[1.13224864e+00,1.27502549e+00,1.08606064e+00]],[[2.43795469e-01,-4.71749865e-02,5.92135966e-01],[4.20458525e-01,4.41429704e-01,5.43252468e-01],[-1.91136543e-02,-2.48030871e-01,-6.02770090e-01]],[[-3.85070771e-01,5.73631883e-01,2.67836481e-01],[3.49490911e-01,3.23592305e-01,-1.06036544e+00],[-1.64631009e-02,-2.48762310e-01,-2.48685256e-01]],[[-3.55986983e-01,6.86804235e-01,6.58866093e-02],[-4.17855561e-01,3.44348252e-01,-5.19454956e-01],[2.82963272e-03,2.74507646e-02,1.35972619e-01]],[[-1.15495220e-01,-7.34331682e-02,5.61940253e-01],[9.40333381e-02,-4.57135975e-01,2.76968867e-01],[-1.88876837e-01,4.53101844e-01,2.31889158e-01]],[[-3.93943280e-01,2.66539246e-01,-1.70937523e-01],[-1.61303163e-01,4.50195879e-01,2.92833205e-02],[-2.31148317e-01,-2.76890039e-01,4.73460793e-01]],[[5.58864653e-01,4.55306172e-02,3.53465229e-01],[-1.39080375e-01,7.41768003e-01,-3.54407281e-01],[-4.60601389e-01,1.81435466e-01,9.49832559e-01]],[[-4.45150286e-01,-4.76691988e-04,5.49541473e-01],[1.36186421e-01,4.88137662e-01,-4.02934812e-02],[-1.43076226e-01,2.52840638e-01,5.51727414e-01]],[[-8.86484757e-02,-7.57239699e-01,-3.14472765e-01],[-2.08822072e-01,4.80376303e-01,5.55589855e-01],[-8.47572327e-01,7.47604892e-02,-3.79565284e-02]],[[-7.89913237e-01,-5.63005924e-01,-1.00302494e+00],[-1.06591058e+00,2.28774950e-01,-1.52754098e-01],[-7.38676250e-01,-6.46535039e-01,-1.47330415e+00]],[[6.64223209e-02,7.66105294e-01,1.92221567e-01],[2.17929423e-01,8.99911150e-02,-2.98046097e-02],[4.30091262e-01,-6.57371759e-01,-2.68561333e-01]],[[8.23953211e-01,3.10965061e-01,-2.74020731e-02],[-2.16869265e-01,-1.68218301e-03,-2.96797186e-01],[-2.16407746e-01,5.23468733e-01,-3.89972210e-01]],[[2.72842348e-01,1.57271475e-01,-1.30444184e-01],[1.56561986e-01,-3.40453595e-01,-9.84043255e-02],[-4.96049702e-01,-5.56519404e-02,-9.16781008e-01]]],[[[-0.33851874,-0.40952945,-0.38767865],[-0.27034152,0.46035022,0.46653026],[0.7119442,0.2791506,0.48762426]],[[-0.6867635,0.2556448,-0.46680984],[-0.38198653,-0.45155257,-0.11516337],[0.11148964,-0.22135542,-0.01898871]],[[0.22245857,0.6628626,0.21240802],[-0.14021304,0.04489559,0.3913841],[-0.15209131,-0.10636427,-0.3893244]],[[-0.35425135,0.20582972,0.13841864],[-0.5666981,0.24624363,-0.00588699],[-0.19743364,-0.5446503,-0.48379856]],[[-0.01967075,0.00292554,-0.95078176],[0.00717176,0.2065381,-1.3710859],[0.09905217,-1.0276077,0.10441198]],[[-0.2255563,-0.01543994,0.16925427],[0.19865575,0.21564934,-0.6671823],[0.8861749,-0.34551305,-0.7796335]],[[0.67447424,-0.2801179,-0.50706476],[-0.10545054,-0.07686323,-0.40440807],[1.0134833,-0.5613098,-0.59759873]],[[-0.94444895,0.19693047,0.3962413],[0.188492,0.05959914,1.0132985],[0.28232175,0.99481267,-0.18436003]],[[-0.23665386,0.28886193,0.5591955],[0.5231854,-0.5111418,0.06554019],[0.27066305,0.41063255,-0.572499]],[[-0.31698078,0.19387068,-0.3944945],[-0.5526054,-0.09812534,-0.353292],[-0.66823363,0.52567345,0.40755782]],[[-0.23534793,-0.27010152,-0.38870555],[-0.69467527,-0.43892625,0.54287887],[-0.03305008,0.02119717,-0.19009846]],[[-0.4764447,1.1372851,-0.6323199],[0.3021197,-0.04035207,0.07363439],[-0.9442253,-0.05021529,-0.41996595]],[[0.71743983,-0.04311537,0.10091006],[0.5656966,-0.04830462,0.7270121],[0.07266032,-0.13200961,-0.15571928]],[[-0.00228176,0.43971765,1.105575],[-0.3293823,-0.01280278,0.16853443],[-0.46303684,-0.557414,-0.06598691]],[[0.04891633,-1.053761,0.5568813],[0.21214424,-0.07904574,-0.35981262],[-0.19846591,-0.04060769,0.51324004]],[[-0.31841266,0.22590393,-0.23629956],[0.5259443,-0.08379743,0.45008558],[-0.17337988,0.14555065,-0.2510643]],[[0.610772,0.72813886,-0.25232115],[-0.5978394,0.04936891,0.6830064],[0.10549631,0.04080164,0.22077203]],[[0.23421282,0.51255196,-0.5527454],[0.28272688,0.04389719,-0.6305609],[0.54872334,-0.31747067,0.19379461]],[[0.49802473,-0.06382836,0.10000577],[0.568536,0.08412497,0.6106586],[0.12660013,0.05940996,1.0520632]],[[0.10728833,0.06740085,-0.38431975],[0.26715776,-0.39691964,0.68715096],[-0.48085216,0.18781611,-0.385074]],[[-0.7165308,0.5629124,0.5531313],[0.27778795,-0.4538581,-0.05694104],[-0.98847806,0.4494554,-0.28278536]],[[0.3191546,0.57162887,-0.2818725],[0.5937156,0.05259655,-0.1552086],[0.36596084,-0.1391189,-0.4319803]],[[-0.17421484,0.04253823,0.48908725],[0.75525945,-0.35173082,-0.07241158],[0.25641912,0.20272973,-0.34532642]],[[0.45849568,0.8836542,-0.38150403],[0.16600473,-0.60974777,-0.17920129],[-0.34740296,-0.94217736,0.23783912]],[[0.49489024,0.06752697,-0.37804985],[0.35157046,-0.5622766,0.7873067],[0.11291934,-0.14293796,-0.05446581]],[[0.04679415,0.33506462,-0.07860086],[-0.5540983,0.22792433,-0.49147126],[0.29857248,0.8589152,-0.9940458]],[[0.6727533,0.12512626,-0.45341563],[0.20946984,-0.6936077,0.33390915],[0.48012322,-0.55173314,-0.06904364]],[[-0.06568064,-0.25982317,0.23936161],[-0.6795823,0.190714,-0.17726956],[1.2066791,-0.14754452,-0.32236522]],[[0.2703276,0.1983519,-1.1778291],[0.4443942,0.18642585,0.13633065],[0.6680002,-0.03582089,-0.6877221]],[[-0.37125027,-0.127736,0.7200729],[-0.59411,-0.16612211,0.271456],[-0.04500032,0.14677818,1.0821838]],[[0.5753841,-1.2058792,0.05220475],[0.19219162,0.09769752,0.22805892],[0.01952208,0.11296283,-0.24387082]],[[0.01417285,0.75192803,-0.08510897],[-0.35165417,0.17056541,0.35636216],[1.7688116,0.57974076,0.3354564]],[[-0.05347209,-0.25106612,-0.72711855],[0.13498832,-0.37724265,-0.04771123],[-0.3512687,0.11626428,-0.30860856]],[[0.16278106,-0.08142002,0.15671204],[0.1450129,-0.05316055,0.5100541],[0.6003634,-0.7771779,0.04720092]],[[0.05458239,-0.83009255,-0.2617527],[0.27907905,-0.00673482,0.36395302],[0.01942431,-0.50325114,-0.4213097]],[[-0.00501589,-0.5727191,-0.01412305],[0.40326777,0.22089823,0.4513268],[0.1549007,0.17255549,0.7320884]],[[0.03828594,0.25292963,-0.29556242],[-0.20202145,-0.26156664,0.63684434],[0.7527255,0.2934654,0.83049226]],[[0.35879076,-0.5382454,-0.17344536],[-0.08439782,0.78465563,0.45765764],[-0.36387396,-0.656327,-0.33243704]],[[0.88307595,-0.18517193,0.36813387],[0.3677962,-0.07651068,0.26772076],[-0.1430085,-1.0486181,-0.56384087]],[[-0.51220274,-0.2741614,0.36959782],[-0.10963191,-0.44662866,-0.03440501],[0.15338588,-0.72044003,-0.56333584]]],[[[-1.19670451e+00,-2.18297303e-01,-1.08647263e+00],[-1.09451604e+00,1.41542017e-01,1.71362236e-01],[-5.55237353e-01,-2.88989842e-01,1.36832595e-01]],[[2.78498948e-01,4.45585251e-01,-5.14005780e-01],[2.44108349e-01,-3.12153138e-02,-3.52232963e-01],[4.95289177e-01,4.45370786e-02,-3.44143540e-01]],[[4.52790797e-01,1.21567339e-01,6.30373240e-01],[3.84449005e-01,-2.09602997e-01,-2.94037968e-01],[7.10954741e-02,3.73419002e-02,4.54451889e-01]],[[1.09829977e-02,8.60154629e-02,8.35026875e-02],[1.82637006e-01,-4.55471396e-01,-4.60727997e-02],[3.37097615e-01,1.55485138e-01,-3.25368971e-01]],[[-2.02478558e-01,3.11087631e-02,-5.45662045e-01],[2.30989620e-01,3.03269736e-02,-5.15920520e-01],[-2.68596131e-02,1.44331634e-01,-3.99452657e-01]],[[-2.25144401e-01,-2.02899631e-02,7.95193762e-02],[-6.96541667e-02,1.46909833e-01,3.51708382e-01],[-3.37206393e-01,-8.27620402e-02,2.56292880e-01]],[[4.31656569e-01,6.47234917e-01,1.22894488e-01],[-3.93289119e-01,3.09183478e-01,4.22137231e-01],[-7.68422484e-02,-3.31067652e-01,1.10443398e-01]],[[1.20461799e-01,-2.15413675e-01,-2.88489103e-01],[2.38011271e-01,-1.67694360e-01,3.61076087e-01],[-7.11859837e-02,-6.54783964e-01,-3.49426001e-01]],[[4.37606931e-01,-1.50605932e-01,3.09849203e-01],[3.31780408e-04,-3.63547802e-01,2.80548871e-01],[6.09772444e-01,-6.29406691e-01,-1.76904961e-01]],[[1.04698524e-01,8.94208997e-02,2.27539837e-01],[3.52614939e-01,-5.00013053e-01,-1.32603079e-01],[4.51478124e-01,-1.02652058e-01,2.57034332e-01]],[[7.33130649e-02,-9.04221460e-02,-4.14499640e-02],[6.34256661e-01,-7.40719307e-03,-7.78661907e-01],[4.20866996e-01,-3.68943624e-02,-5.38617551e-01]],[[-1.64577097e-01,3.04633901e-02,-1.29206601e-04],[-1.00318241e+00,-2.86613882e-01,-1.88498780e-01],[1.96389593e-02,-1.72846258e-01,1.63764551e-01]],[[4.91588652e-01,4.08481151e-01,7.19291642e-02],[7.02580586e-02,-9.53068674e-01,-1.10145867e-01],[1.71459258e-01,-3.06634635e-01,1.49022624e-01]],[[-5.58971524e-01,1.36948049e-01,-2.67788410e-01],[-6.85565174e-01,-4.15262789e-01,-1.28649190e-01],[-1.41795766e+00,-7.69685924e-01,-7.16129363e-01]],[[-7.06570029e-01,-3.83631110e-01,-6.34368658e-01],[4.00298715e-01,-1.58758033e-02,-5.19605339e-01],[2.33123645e-01,5.77250540e-01,4.48928118e-01]],[[4.02619243e-01,2.54162103e-01,1.51403278e-01],[-3.47136855e-01,4.02386159e-01,-6.29652619e-01],[-3.15449625e-01,7.07355857e-01,-2.89343577e-02]],[[2.84986824e-01,-1.12624548e-01,-1.18147753e-01],[2.66156286e-01,-1.65031344e-01,-3.89821082e-01],[5.07284999e-01,2.34211907e-01,-3.70862991e-01]],[[3.99823368e-01,2.64070511e-01,-1.02273963e-01],[-1.88655317e-01,1.42184598e-02,-6.92900479e-01],[-7.64060438e-01,3.29403579e-01,-1.19514756e-01]],[[-3.87553722e-02,-8.13800931e-01,6.32949412e-01],[-1.19642481e-01,-1.91748515e-01,-2.19782948e-01],[7.66348600e-01,2.26531893e-01,7.98136890e-01]],[[-1.93199307e-01,4.19122547e-01,3.49345803e-02],[4.93961304e-01,5.38637042e-01,1.03707969e-01],[1.10273993e+00,3.62531215e-01,1.67137754e+00]],[[1.75595377e-02,1.50407538e-01,-1.08266854e+00],[-2.24881604e-01,4.00450110e-01,-2.20401824e-01],[-5.50664902e-01,-4.99339461e-01,-2.03679726e-01]],[[7.49009550e-01,3.05137187e-01,-5.01142703e-02],[2.66485780e-01,4.79560673e-01,-2.59824902e-01],[-1.82536766e-01,-4.39946741e-01,2.17885599e-01]],[[3.88857722e-01,1.75550833e-01,2.38977909e+00],[-1.17246032e-01,-1.64119676e-01,1.78409621e-01],[-9.00421083e-01,2.80106328e-02,4.60311204e-01]],[[-6.43173814e-01,2.05740258e-01,-6.57350600e-01],[4.04073715e-01,-2.84178376e-01,2.69828558e-01],[-3.90811592e-01,-8.78357738e-02,5.17736554e-01]],[[6.27890706e-01,3.39152843e-01,6.55297518e-01],[6.14034235e-01,-9.16674510e-02,2.45518848e-01],[9.92560804e-01,-2.89076000e-01,8.33389521e-01]],[[1.55788705e-01,-3.92588302e-02,-1.02436244e-01],[4.85154152e-01,2.35842884e-01,1.82493985e-01],[-3.26975882e-01,6.28632247e-01,-5.96950293e-01]],[[-2.31170312e-01,3.16547394e-01,6.79165304e-01],[3.31557453e-01,-7.12883472e-02,3.33008200e-01],[5.55198431e-01,5.49008884e-02,3.96219999e-01]],[[-2.15377733e-01,-2.30660319e-01,-2.00080395e-01],[-1.12477911e+00,2.88919657e-02,-8.31785500e-02],[-3.45260829e-01,3.74041259e-01,-2.87016600e-01]],[[-4.37723309e-01,3.84390384e-01,-2.14889860e+00],[3.10224324e-01,-1.40136242e-01,-7.86570847e-01],[-7.50644803e-02,-3.21337059e-02,-2.05063060e-01]],[[-3.69961113e-01,-3.35403346e-02,-2.36829013e-01],[-1.11280821e-01,-2.63845026e-01,-3.91111463e-01],[-2.31034845e-01,4.16174345e-02,-5.77801585e-01]],[[2.15557531e-01,2.97770172e-01,1.81322426e-01],[1.16830468e-01,5.35945296e-01,-1.58854216e-01],[-3.92930686e-01,6.11259878e-01,4.48029861e-02]],[[-5.80336750e-01,-3.86206694e-02,-1.96542650e-01],[4.52771783e-01,5.71939170e-01,3.52266252e-01],[4.28561866e-01,-4.42054063e-01,-1.54309109e-01]],[[4.67172265e-01,2.20336989e-01,-2.14491412e-01],[3.08365762e-01,-5.77405512e-01,1.28968611e-01],[2.43079260e-01,-4.67929654e-02,-1.93170220e-01]],[[8.31097305e-01,-1.25170231e-01,-1.52426898e-01],[5.47198504e-02,6.34048462e-01,1.07013494e-01],[2.17276692e-01,-8.08468014e-02,-6.69000447e-01]],[[-5.34734130e-01,-9.08776820e-02,2.74086773e-01],[-4.54185188e-01,-1.56232476e-01,-1.88846532e-02],[-4.88933355e-01,4.51015174e-01,-9.46447432e-01]],[[5.96975610e-02,8.83198082e-02,-3.53271142e-02],[-9.48505998e-02,1.97230130e-02,5.46683609e-01],[5.57324886e-01,-5.90642154e-01,4.18647826e-01]],[[1.88325658e-01,1.70930743e-01,-3.15299988e-01],[4.04740810e-01,-3.39810997e-01,-2.99339443e-01],[1.04095705e-01,6.96719348e-01,2.68819809e-01]],[[2.37361804e-01,7.85617307e-02,1.36103079e-01],[6.87381089e-01,5.89316487e-01,-2.53767759e-01],[7.25050867e-01,6.70474023e-02,-7.39718750e-02]],[[-1.08050215e+00,4.28134203e-01,1.47314556e-02],[-3.29653919e-01,-2.88887739e-01,5.19781232e-01],[-9.61196497e-02,-4.28897619e-01,1.16485141e-01]],[[3.16129327e-01,-3.67405027e-01,2.54738063e-01],[6.13253236e-01,-2.18992367e-01,-5.71832895e-01],[-4.13175374e-01,3.98733079e-01,1.29029706e-01]]],[[[0.43591484,0.26185566,0.65701306],[-0.1678485,0.19509424,0.02268431],[0.09517399,0.04779746,0.2244148]],[[-0.07675659,0.62213105,0.4323477],[-0.3380448,-0.04638378,-0.39763921],[-1.1623242,-0.4443574,0.37206632]],[[-0.20647393,0.81540567,-1.8968537],[0.18671842,-0.53138644,-0.20598322],[-0.7689332,0.5648177,0.12419201]],[[0.67157423,0.5393685,0.27020502],[-0.19678175,0.1787601,-0.37636402],[-0.2698932,-0.27832136,0.22182405]],[[-0.23030171,0.14517365,0.02057634],[0.31344387,0.69764155,-0.2564064],[0.38553953,-0.5614943,-0.10575008]],[[-0.24529678,-0.3096045,-0.11802337],[0.34904546,-0.02307945,-0.23225598],[-0.20906907,0.3360222,0.49076504]],[[0.08841423,-0.3433973,-0.19098747],[0.3549643,-0.5348841,0.32507515],[0.3309429,-0.7229042,0.08579522]],[[0.337297,0.6457998,-0.30734122],[0.13347866,0.04859035,-0.2308342],[-0.45044702,0.8203732,0.25306296]],[[0.36918426,-0.5941392,0.14766501],[0.25648355,0.06271426,0.12827927],[-0.29341716,-0.60256433,0.2148317]],[[0.06417944,0.13577572,-0.19635788],[-0.31372333,-0.09940397,1.0137308],[-0.47626796,-0.08835862,0.60535425]],[[-0.68579745,0.41544428,-0.1952265],[-0.04396521,-0.05624685,0.2216285],[-0.32515448,0.18691649,0.11105737]],[[0.06506291,-0.28122312,-0.49920976],[-0.13956808,0.43850735,-0.2588779],[0.09116154,0.25113523,-0.21873145]],[[-0.03131674,0.9603422,0.79302114],[-1.0251929,-0.17572968,-0.48357576],[0.15089141,-0.1156173,-0.6618652]],[[-0.5579664,-0.39598888,0.08776738],[0.03521721,-0.8860779,0.24591273],[-0.6668025,0.09279659,-1.0047195]],[[-0.46904305,0.26091048,-0.3374241],[-0.18327846,-0.84455127,-0.0631642],[0.5424555,0.74307823,0.22343495]],[[-0.20656839,0.5299913,0.21992277],[-0.13203579,-0.45155165,0.02343708],[-0.39234412,0.04063935,-0.10508136]],[[0.13989656,0.34123844,0.09510232],[0.17218076,0.4041635,-0.0226915],[0.2349725,0.01355565,-0.11432472]],[[-0.66917735,-0.15859123,-0.95313287],[-0.17704281,-0.41793954,0.10960244],[0.15891886,0.27076373,-0.31366217]],[[0.658485,0.21601382,-0.13563621],[-1.3283513,-0.5966194,0.7602105],[-0.7463866,0.1658527,0.32800195]],[[0.8763462,-1.0067775,0.13698426],[-0.9718578,-0.4911621,0.35863262],[0.6594993,-0.22372216,-0.2291188]],[[0.34727645,-0.59560597,-0.90761715],[-0.682919,0.06734384,0.19023548],[0.17478403,-0.16939658,0.10379936]],[[1.2563494,0.98608315,0.63638264],[0.00606847,0.3380615,0.44350523],[0.08707156,0.50197613,-0.4072715]],[[0.02431604,-0.23321675,0.6954983],[-0.3132809,0.11118357,0.20027626],[-0.27974,0.20917386,-0.3009658]],[[-0.6017091,0.47970623,0.42670044],[0.14978935,-0.09493881,0.92358863],[0.25076786,-0.20618941,0.22822703]],[[0.27218062,-0.44198546,0.2675527],[0.20378372,-0.85438544,-0.3962774],[0.3601321,-0.3315827,0.19404447]],[[0.7432288,0.18337262,0.30111322],[0.29637158,0.2895826,-0.7984341],[0.3551398,-0.7408481,0.870707]],[[-0.07641665,-0.28122982,0.50288326],[0.33962187,-1.3100532,1.1179587],[0.22606952,-0.16338035,-0.3067939]],[[0.20643382,-0.4247319,0.37593257],[-0.11477637,0.53704417,0.25824708],[0.26376733,0.26881427,0.5876369]],[[0.02333411,0.27038872,-0.78896725],[0.0867432,0.01270163,-0.93075484],[0.14424992,-0.05061355,0.10309562]],[[-0.12763532,-0.21906082,-0.69934887],[-0.6815059,-0.20777659,0.12799473],[-0.86784387,-0.49789342,0.3567744]],[[0.46449018,-0.24309829,0.16029362],[-0.07045385,-0.2868529,0.26577997],[0.22377172,0.45180744,-0.50396717]],[[0.94553673,-0.27829364,0.00710663],[-0.06048771,-0.24156114,-0.2538924],[-0.261043,-0.57705194,-0.18240108]],[[0.21921824,-0.85985667,0.5993008],[0.5282493,0.07419773,0.28681952],[-0.18982653,-0.10184901,0.21150884]],[[-0.01801026,-0.16755357,-0.31743905],[0.42143175,-0.3788953,0.98051554],[0.3202168,0.01313881,0.39991012]],[[0.7347726,-0.6283771,-0.66312575],[0.17170593,0.0815654,0.85028726],[-0.34985125,-0.42769188,-0.13471356]],[[0.45556608,-0.22274554,-0.02927371],[-0.05815328,-1.0405493,0.63554573],[-0.08941171,0.20808198,0.06436213]],[[-0.7720149,-0.42678225,0.3330293],[-0.6610438,0.7152999,-0.81001717],[0.53438926,0.49348944,-1.4290584]],[[-0.01139501,-0.3792557,-0.13858193],[-0.3158285,-0.75494665,0.86457145],[0.2423205,-0.22263415,-0.39038342]],[[0.46818697,0.21995269,-0.04657048],[0.31938902,-0.09268851,-0.48269245],[-0.5235705,0.06164872,-0.02129551]],[[-0.73736066,0.63021886,-0.7136958],[-0.45306495,0.17556854,0.7352265],[0.1030875,-0.23246366,-0.34509775]]],[[[0.20560162,0.13912432,0.1695565],[0.02575312,0.31486252,0.02269533],[-0.42393884,-0.0910041,0.29629138]],[[-0.5409706,0.41490135,0.05715256],[0.5027163,0.18599194,-0.28975686],[-0.3197535,-0.58791846,-0.06537491]],[[-0.90663594,-0.0367971,0.17054234],[0.22999248,0.36644834,0.6219835],[-0.5356405,0.21821249,0.15103355]],[[0.36166373,0.6114197,-0.2775764],[0.19489413,0.9232817,-1.0783435],[-0.02512514,-0.35736737,-0.17221169]],[[0.21227272,0.02754485,-0.3230453],[0.15628871,-0.03245817,0.52903765],[-0.33378503,0.1569745,0.97021246]],[[0.31321943,-0.7176269,0.2778832],[0.4050765,-0.56660795,-0.17637098],[0.33494303,-0.4140127,1.1656733]],[[-0.00395404,0.96687126,-0.66346335],[0.44463083,0.6921404,-1.0087306],[0.07576966,-0.0273692,0.11331362]],[[0.6864108,0.33662623,-0.1184188],[-0.9422452,-0.54391104,0.06384262],[-0.17588463,0.16063036,0.11582147]],[[0.5113823,-0.54598063,0.14746629],[-0.00733137,-0.02879532,-0.4472632],[0.9370523,-0.06257463,0.41274598]],[[0.03260526,-0.15628578,-0.7805723],[-0.55713665,-0.6331485,-0.14031665],[0.05321867,0.26709685,-0.26647994]],[[0.9751431,0.07001057,-0.05982264],[-0.32946277,-0.9522335,-0.43148875],[-0.54118156,0.28353652,0.19357459]],[[-1.075518,-0.33317274,1.0745623],[-0.5106284,0.33954105,0.7372899],[-0.308116,0.10316054,0.6009863]],[[-0.68111074,0.1348756,-0.2511559],[0.84064215,0.15821715,0.10191041],[-0.26475018,0.46662593,-0.19036569]],[[-0.28149796,-0.5389092,-0.41942775],[-0.25809294,-0.03035521,-0.59649765],[-0.2430893,-0.47509983,-0.321348]],[[0.03128376,0.23201574,-0.3165937],[-1.1413203,-0.4264796,0.13611881],[0.4489863,-0.36727303,0.02383703]],[[0.04650869,0.4153568,-0.48852703],[-0.03835524,-0.35666853,0.4381555],[0.0894724,-0.30431622,0.26996702]],[[-0.45784554,-0.94002205,0.39382312],[-0.606856,-0.24052891,-0.5517218],[-1.0954319,-0.17878802,0.45624343]],[[-0.2929202,0.7066407,0.00951967],[0.80143017,0.2592237,-0.04582087],[-0.46789616,0.24755833,-0.5552888]],[[-0.7194734,-0.18786353,-0.19212805],[-0.7827119,-0.85782534,0.16206488],[-0.617608,0.3936092,1.0007392]],[[0.27380934,0.22228184,-1.143039],[-0.07126311,0.27028146,-0.00701187],[0.09165636,-0.6278205,0.26468414]],[[0.42217273,-0.25293505,-0.2778005],[0.39584324,-0.15484603,-0.6362291],[-0.08491199,-0.29630092,-0.38594234]],[[0.63505924,-0.31373826,0.2521194],[0.06366168,-0.18474853,0.1346483],[-0.5772141,-0.55070585,0.92795575]],[[-0.48595977,-0.2266488,0.16534963],[-0.20325126,-0.1441884,-0.42660317],[-0.34635857,-0.05254712,0.4497392]],[[-0.6168325,-0.3436259,-0.21848913],[-0.08423232,-0.07779257,-0.11570214],[-1.0923994,0.49788108,0.48837063]],[[0.19152702,0.57219666,0.16867374],[-0.0381279,-0.21551132,-0.3727313],[-0.2680228,0.2694657,-0.12463348]],[[-0.31238517,-0.3903049,0.65019274],[-0.5963139,0.307485,0.21079074],[0.25156692,-0.44158614,-0.07057121]],[[0.04186297,0.49069974,-0.87505573],[0.52945614,-0.29630592,0.39102408],[0.16402842,-0.4571819,0.65913945]],[[0.16276743,0.1618095,0.04453017],[-0.90718365,0.62430215,-0.05209409],[0.9669474,0.49881622,0.2573941]],[[0.18651342,-0.67593724,0.47506002],[0.43668664,0.20149066,-0.6645086],[-0.03337199,0.08545741,-0.06439269]],[[-0.36391613,-0.02670389,0.13538486],[-0.11870542,-0.43589002,0.18177472],[0.5938594,0.2815012,0.11180261]],[[0.36788294,-0.11704686,-0.02005322],[0.05435233,0.23334323,-0.81107324],[-0.42299086,-0.51465225,-0.43355507]],[[0.1395989,0.70262957,0.2367721],[-0.51831603,0.2038986,1.1898367],[-0.02332265,-0.46782774,0.5969861]],[[-0.15863939,-0.13780586,-0.34214953],[-0.76740605,-0.0837559,-0.8205857],[-0.28933385,-0.5736617,-0.35839006]],[[0.27419975,0.28568426,-0.1558242],[0.9672646,0.08330736,-0.2802803],[0.73471874,0.05677541,0.73841834]],[[0.2152385,-0.01464298,0.5925794],[0.27011177,0.20095783,-0.2799898],[0.509662,0.18842246,0.3504294]],[[0.21634474,0.4396716,-0.397695],[-0.60089594,-0.2926666,-0.11881205],[-0.94611675,0.70862126,-0.49258292]],[[0.7097177,-0.25691918,-0.24732287],[1.0020243,0.32756376,-0.22142342],[0.06712801,-0.2893053,0.02863089]],[[0.5790284,0.4809503,-0.08326568],[0.19114551,0.32079175,-0.38728374],[0.44623518,0.52088416,0.8292365]],[[-0.44108403,0.33455122,-0.01046348],[0.4637095,-0.28238508,-0.6347419],[-0.0448978,0.14155893,0.2845251]],[[0.05957773,0.21522263,-0.01061532],[-0.15700689,-0.44779193,-0.21335995],[-0.08257703,-0.33279568,0.07015378]]],[[[-1.65204084e+00,7.13744700e-01,4.97080773e-01],[-7.11763382e-01,3.67948055e-01,3.33702952e-01],[-2.63603449e-01,3.78987014e-01,3.56219649e-01]],[[9.17852759e-01,5.73978007e-01,-1.02404490e-01],[1.00741518e+00,-1.53516814e-01,2.26691529e-01],[9.86213326e-01,1.78610578e-01,7.58037627e-01]],[[-3.30984414e-01,1.34394214e-01,-5.37565589e-01],[1.22842073e-01,2.16308758e-01,2.92536527e-01],[4.61073108e-02,-1.19822323e-01,-3.81871164e-01]],[[-2.45205015e-01,3.42259556e-01,9.03612152e-02],[-1.04518032e+00,3.70836444e-02,-1.33783996e-01],[2.35078916e-01,-3.58006090e-01,8.94397497e-03]],[[-2.02741325e-01,-1.35083303e-01,-2.98561186e-01],[-9.08423483e-01,-6.12764359e-01,1.22860827e-01],[5.29002547e-02,-3.64160717e-01,-7.72519052e-01]],[[3.76932253e-03,-4.18528348e-01,3.00382286e-01],[-3.69920075e-01,-1.66789412e-01,9.14823636e-02],[-4.04265910e-01,1.00834370e-01,2.55583793e-01]],[[-5.73717952e-01,-3.21084589e-01,-2.96817929e-01],[-2.03333393e-01,-4.57212985e-01,-4.97877687e-01],[3.00613582e-01,6.34442091e-01,-4.89672087e-02]],[[2.67160684e-01,-2.48819858e-01,5.06496876e-02],[3.27037841e-01,-1.68705761e-01,-7.08509505e-01],[5.43197952e-02,-1.30167931e-01,6.07081890e-01]],[[3.25553626e-01,1.15298137e-01,-1.97633188e-02],[-4.33595091e-01,-3.17728102e-01,-2.91714489e-01],[4.57313210e-01,-3.33144441e-02,4.35469419e-01]],[[1.40075362e+00,-2.98156887e-01,-8.89525890e-01],[4.37621325e-01,5.08865893e-01,4.15894747e-01],[4.09027964e-01,5.42667270e-01,3.88591051e-01]],[[4.59593385e-01,-1.62076488e-01,1.45254552e-01],[-1.54569939e-01,1.83353439e-01,3.97825360e-01],[3.39099228e-01,5.28462470e-01,2.30632365e-01]],[[-4.58780266e-02,-3.52107018e-01,-9.55763161e-01],[2.89263248e-01,7.35109672e-02,3.25928330e-01],[-4.70930010e-01,8.01757351e-02,-5.85632175e-02]],[[5.01342654e-01,-1.47239760e-01,1.94525436e-01],[-4.23614115e-01,9.01324376e-02,-1.19643942e-01],[-5.33290565e-01,-8.51999283e-01,8.28654110e-01]],[[8.05117428e-01,-1.79329649e-01,3.94684792e-01],[-5.37414610e-01,-8.16126168e-02,-2.06437126e-01],[-7.14423060e-01,2.52658606e-01,1.07606876e+00]],[[-1.80861339e-01,-3.18015426e-01,-7.28443325e-01],[-3.77869129e-01,5.72550237e-01,3.01741920e-02],[2.17282940e-02,-2.39143208e-01,-8.23182881e-01]],[[-3.32307220e-01,-1.50802523e-01,2.15875693e-02],[2.19908938e-01,-4.44076598e-01,2.49142513e-01],[-1.56636059e-01,9.13418531e-02,-5.10042667e-01]],[[5.18608451e-01,-3.50151747e-01,9.13719684e-02],[3.64529759e-01,-8.44708979e-02,-4.57845300e-01],[1.07253540e+00,-2.16588229e-01,5.31738102e-01]],[[-9.70449507e-01,1.13215551e-01,6.99907020e-02],[5.21020055e-01,8.52472112e-02,-1.18418917e-01],[-4.96746749e-02,2.77990192e-01,-7.37982333e-01]],[[-1.11503102e-01,-2.54073173e-01,-8.43484551e-02],[-3.65312666e-01,-3.71257782e-01,6.90614223e-01],[1.09915137e+00,1.00089812e+00,7.58144975e-01]],[[-6.01490662e-02,-1.62818551e-01,-1.01210065e-01],[-2.72044480e-01,3.75137955e-01,3.41519833e-01],[7.79752672e-01,-3.24851334e-01,1.09612018e-01]],[[7.81340182e-01,6.51831105e-02,-3.56841296e-01],[-2.01507598e-01,-4.59000051e-01,5.07358015e-01],[2.20326990e-01,-2.39114240e-01,8.43610093e-02]],[[-4.46913064e-01,-2.84889013e-01,4.96254444e-01],[1.21353336e-01,-3.27435106e-01,-1.66418806e-01],[8.68419528e-01,3.45451534e-02,-4.03650813e-02]],[[4.97823536e-01,4.17627215e-01,6.58765852e-01],[1.50510356e-01,-2.71259993e-01,4.49426323e-01],[-6.83939576e-01,4.99498636e-01,6.37166917e-01]],[[-6.16145618e-02,-1.55237764e-02,-2.97789127e-01],[-1.21330112e-01,-6.03092194e-01,-3.67702637e-03],[-2.09656879e-01,-2.19806403e-01,1.21796288e-01]],[[2.34487996e-01,-4.24965739e-01,-3.60476144e-04],[-4.47585136e-02,-2.91540414e-01,1.17856652e-01],[2.19145060e-01,-2.29237005e-01,3.39831352e-01]],[[-3.36600989e-01,-1.20178461e-01,5.05492687e-01],[-9.45301592e-01,-1.05555750e-01,-8.02372336e-01],[1.44613075e+00,4.77768868e-01,-4.28008616e-01]],[[2.12545559e-01,-4.54465412e-02,1.62476301e-01],[-8.41700435e-01,6.60750121e-02,4.09995079e-01],[1.69773579e-01,-2.78688129e-02,2.47427270e-01]],[[-3.09041619e-01,6.03165925e-01,-8.29926878e-03],[-4.07289356e-01,4.47706103e-01,2.37755805e-01],[1.95126474e-01,-5.27580857e-01,-1.90144420e-01]],[[-1.66257739e-01,-2.52179116e-01,1.51184825e-02],[-6.57507062e-01,1.65928870e-01,-1.31052524e-01],[-2.50322700e-01,-4.59910572e-01,-3.94873887e-01]],[[-1.34047461e+00,6.15528047e-01,-1.42557576e-01],[-7.35198438e-01,1.08754843e-01,-3.85482423e-02],[7.44019672e-02,1.39401376e-01,-9.40580368e-02]],[[-8.98027062e-01,8.62808406e-01,5.97794294e-01],[-7.80498266e-01,4.94147867e-01,-1.21713534e-01],[-3.61353636e-01,8.72240141e-02,4.13467772e-02]],[[-3.82834643e-01,-1.45449936e-01,-2.19785288e-01],[-4.08942491e-01,-3.88115346e-01,-3.33320737e-01],[2.90224627e-02,3.36735874e-01,-4.08007324e-01]],[[5.45398176e-01,1.00303972e+00,5.65270483e-01],[2.61558861e-01,-6.66199550e-02,1.12068367e+00],[5.78144729e-01,-1.07317083e-01,1.37830704e-01]],[[8.67898822e-01,4.32342321e-01,2.12770358e-01],[2.45350838e-01,-1.02650307e-01,-5.31246603e-01],[5.76485991e-01,-4.87493053e-02,-2.17075422e-01]],[[-4.96437579e-01,-1.21126413e-01,-3.15297514e-01],[-2.07424626e-01,-4.84630644e-01,2.52397954e-01],[2.00870469e-01,3.01149696e-01,-7.22718179e-01]],[[3.74052584e-01,-6.30006269e-02,6.40004158e-01],[-3.55005890e-01,9.96053591e-02,1.09846103e+00],[4.57739830e-01,3.73043604e-02,1.79819018e-01]],[[-9.08327624e-02,-9.22649875e-02,-2.79068314e-02],[4.74010594e-02,2.34648034e-01,1.14595786e-01],[1.34557402e+00,5.23002744e-01,5.37226200e-01]],[[-2.16586098e-01,-1.47194549e-01,2.91538477e-01],[6.21306956e-01,-3.59242946e-01,-5.91931880e-01],[5.01001477e-02,2.34699920e-01,-3.89376998e-01]],[[-4.19405550e-01,-3.50388795e-01,1.04145734e-02],[-7.86783546e-02,-3.21683288e-01,-1.86713159e-01],[-6.24934673e-01,-2.46132404e-01,-1.68972328e-01]],[[9.06924307e-01,3.07424366e-01,-4.20953542e-01],[2.48056993e-01,-2.05648944e-01,5.02560198e-01],[-2.80778766e-01,3.26373428e-01,-4.52640444e-01]]],[[[8.29675436e-01,2.78180182e-01,4.18380171e-01],[-1.70045402e-02,-7.09389150e-01,-5.66079915e-02],[-3.42344314e-01,-6.03777707e-01,-5.63074291e-01]],[[-5.90263069e-01,-7.99857676e-01,1.45900249e-01],[-1.36894315e-01,-2.49537975e-02,-4.39832389e-01],[-1.21303864e-01,3.41627710e-02,3.25871319e-01]],[[5.60592227e-02,-9.47995305e-01,-9.28291202e-01],[4.02135760e-01,-2.90409446e-01,1.21985808e-01],[-1.46417663e-01,-6.19888127e-01,-1.98468074e-01]],[[-6.03622794e-01,1.32837772e-01,4.58796203e-01],[-1.76399231e-01,-5.57760060e-01,-1.22444108e-01],[-3.07116985e-01,-5.08580506e-02,2.68046498e-01]],[[-1.94857389e-01,-6.70132816e-01,-1.64322797e-02],[-1.76650420e-01,-1.44511461e-01,2.63366878e-01],[-1.17791958e-01,9.96546268e-01,-3.75664115e-01]],[[-2.17575863e-01,-1.84408441e-01,-8.77187476e-02],[-7.61445701e-01,-2.74512053e-01,1.65031791e-01],[-6.29436791e-01,3.22800070e-01,-3.78040940e-01]],[[1.05243647e+00,2.88487524e-01,3.54969352e-01],[5.00844359e-01,-1.97243631e-01,2.96806358e-02],[-4.09845501e-01,7.64116943e-01,1.84444577e-01]],[[-2.44569644e-01,-1.80196799e-02,2.44743407e-01],[-2.72778384e-02,-1.80138886e-01,2.39340916e-01],[7.31163740e-01,-1.08328350e-01,7.39282191e-01]],[[-6.21648312e-01,4.26436141e-02,1.76329422e-03],[1.04701176e-01,1.54167011e-01,2.49627605e-01],[-2.89249808e-01,-1.29554331e-01,-6.96592126e-03]],[[-4.30167764e-01,-8.69546294e-01,-3.57155949e-01],[-4.72797185e-01,-2.70723045e-01,3.36454630e-01],[-1.57123223e-01,6.56702042e-01,-3.30703765e-01]],[[-2.11921819e-02,-5.07007182e-01,5.12922049e-01],[-6.73041418e-02,-1.64188802e-01,-3.64136577e-01],[-8.13206062e-02,-1.10722780e-01,1.00642078e-01]],[[7.06388235e-01,1.04886830e-01,-3.03435802e-01],[7.39396572e-01,-7.52335846e-01,-2.28681430e-01],[9.88052785e-01,1.54294208e-01,3.34592015e-02]],[[3.51861380e-02,2.15007037e-01,2.10560858e-01],[-4.18492615e-01,3.26797962e-02,-5.50875902e-01],[8.60158145e-01,4.38665867e-01,1.12910092e+00]],[[2.81588852e-01,6.05028868e-03,1.44278526e-01],[-4.13279384e-01,-3.95035416e-01,5.90303093e-02],[-3.08220834e-01,1.17998429e-01,6.80087954e-02]],[[-3.26348245e-01,-3.06770355e-01,-2.88628250e-01],[-5.20915017e-02,-8.99790823e-02,1.36772841e-01],[-2.85601616e-01,6.07403778e-02,5.30675769e-01]],[[3.65336627e-01,1.09629922e-01,4.65505719e-01],[-5.09159379e-02,5.89179397e-01,1.17352557e+00],[-2.64727995e-02,-5.92578612e-02,-4.11979616e-01]],[[9.40932274e-01,-3.33043009e-01,4.67706239e-04],[9.51715946e-01,-7.14209080e-02,-6.59127653e-01],[-6.69978917e-01,-4.93807197e-01,2.17849314e-01]],[[-3.77541751e-01,-9.15028676e-02,2.80231088e-01],[-4.25595790e-01,3.32113355e-01,-4.85197641e-02],[-5.67578495e-01,7.65771866e-02,3.14860016e-01]],[[3.62650484e-01,-4.30595204e-02,2.37342790e-01],[-9.99278069e-01,1.12257636e+00,3.85865918e-03],[-1.13326311e-01,-7.59933412e-01,5.38793981e-01]],[[-5.55118740e-01,3.79948944e-01,-1.46524878e-02],[1.00260556e-01,4.49910671e-01,-1.02330662e-01],[4.52193707e-01,-4.59507495e-01,7.75761664e-01]],[[1.91923127e-01,-2.43551746e-01,2.80155808e-01],[-5.07635474e-02,-2.05964088e-01,6.03419065e-01],[2.91589677e-01,9.10509378e-02,-3.61549519e-02]],[[-2.69554108e-01,1.78625807e-01,-1.38553694e-01],[-2.52302706e-01,-5.19068958e-03,3.54383081e-01],[-4.03086662e-01,3.99423599e-01,-4.62760264e-03]],[[-2.85838962e-01,2.85895407e-01,-5.34038067e-01],[-5.44874892e-02,-1.06210992e-01,2.79216856e-01],[4.51171964e-01,-5.70056319e-01,-5.88948607e-01]],[[2.02335998e-01,2.61449993e-01,-6.93760663e-02],[-8.10217381e-01,1.21222821e-03,6.67514026e-01],[9.90749419e-01,-9.13156942e-02,-3.14829975e-01]],[[-3.58960062e-01,6.39405921e-02,1.13625355e-01],[5.31255662e-01,-4.82510149e-01,-5.45452356e-01],[3.91709246e-02,-2.49391213e-01,-5.22457242e-01]],[[6.44834757e-01,-2.26581499e-01,-5.59712887e-01],[2.35679463e-01,6.50722384e-01,2.43747592e-01],[-1.01761751e-01,4.18880522e-01,-6.97593391e-01]],[[-2.38743424e-01,-4.51254964e-01,8.75226319e-01],[-7.71265328e-01,1.29024521e-01,-5.85500836e-01],[-2.97549553e-02,-1.29249096e-01,1.10210352e-01]],[[1.88600942e-02,5.25606811e-01,3.21815968e-01],[-6.21404469e-01,3.52364443e-02,1.09480011e+00],[-1.49454668e-01,-2.25799605e-01,-4.65032458e-01]],[[-2.18034878e-01,-1.09025216e+00,-1.23586893e-01],[5.70337474e-01,-2.80701429e-01,2.51166314e-01],[-2.55074084e-01,-4.69146550e-01,6.69100881e-01]],[[-1.18528366e-01,-1.18552214e-02,-7.10051537e-01],[-3.93703759e-01,-4.11919236e-01,-5.95743120e-01],[-1.04110289e+00,-4.65557277e-02,4.53229994e-01]],[[8.14707339e-01,-4.38583076e-01,6.44146860e-01],[8.07150066e-01,1.34960786e-01,-3.73201221e-01],[-5.09774148e-01,-2.37651110e-01,2.64595211e-01]],[[5.55547237e-01,3.10024112e-01,1.97886512e-01],[7.57505149e-02,4.30863500e-01,-2.16808751e-01],[2.22558267e-02,-2.06491813e-01,-4.13123257e-02]],[[-2.64748603e-01,-3.22565287e-01,3.34506810e-01],[-6.81796253e-01,-4.65516686e-01,7.87007213e-01],[5.90327144e-01,6.00403845e-01,-9.76658821e-01]],[[-1.99416459e-01,6.32573962e-01,4.40355897e-01],[4.00687248e-01,1.57151803e-01,-5.68778098e-01],[3.06417912e-01,4.04366940e-01,5.38214028e-01]],[[-4.28507864e-01,-4.64082807e-01,7.78621674e-01],[-4.19378519e-01,1.22905366e-01,1.89726979e-01],[8.39233678e-03,-4.12041277e-01,-5.56421101e-01]],[[-3.25347513e-01,4.40145016e-01,-3.01928967e-01],[3.28405321e-01,-1.34540766e-01,-2.57309854e-01],[3.00804496e-01,1.33532047e+00,2.26872310e-01]],[[6.23275399e-01,6.21946514e-01,-5.15199564e-02],[1.49754807e-01,-1.54110342e-01,1.84600446e-02],[4.94153440e-01,4.83150542e-01,-5.84190227e-02]],[[-2.70681947e-01,-2.80808806e-02,3.13067049e-01],[8.15744042e-01,-1.87985256e-01,-2.71044731e-01],[-7.40089774e-01,5.06848037e-01,-4.10980344e-01]],[[-2.51270235e-01,5.52577138e-01,5.08454591e-02],[8.47980320e-01,2.94879943e-01,1.14346318e-01],[3.29030663e-01,-1.99984550e-01,5.72878957e-01]],[[-3.22453052e-01,-1.05597317e+00,2.14598209e-01],[-6.93109334e-01,-3.07991612e-03,-1.14386775e-01],[-6.43909812e-01,2.96210080e-01,-1.21782824e-01]]],[[[-4.79198456e-01,3.22279125e-01,-9.71014891e-03],[2.69779330e-03,3.03125262e-01,2.52380818e-01],[-8.29571843e-01,8.87962401e-01,-2.76229411e-01]],[[2.17731640e-01,-1.81894243e-01,2.33928919e-01],[8.59123394e-02,-1.15754806e-01,-5.67405403e-01],[-4.34145212e-01,2.61869971e-02,5.73587537e-01]],[[4.28167768e-02,3.03840667e-01,-2.14809462e-01],[-2.39229113e-01,-8.38008150e-02,-1.32428601e-01],[-3.47425848e-01,2.19692096e-01,3.89158040e-01]],[[-8.27085823e-02,-1.56677902e-01,-7.47175574e-01],[1.86723679e-01,6.46290541e-01,6.25601232e-01],[7.63920367e-01,5.26128054e-01,-2.23957539e-01]],[[9.91208777e-02,-1.21967509e-01,3.73140723e-01],[-5.30938148e-01,-3.91302347e-01,-9.81206894e-01],[-3.80244851e-02,3.24788004e-01,3.22325915e-01]],[[-5.31822816e-02,-4.65369403e-01,9.61445212e-01],[1.28120273e-01,2.14914635e-01,-5.12644529e-01],[4.02765989e-01,-7.19620585e-01,-1.07890785e+00]],[[-4.78785425e-01,-2.73947306e-02,2.35011965e-01],[2.51069844e-01,2.61794448e-01,-3.26162159e-01],[-1.84666559e-01,-1.03619742e+00,-5.16149342e-01]],[[2.35703036e-01,2.55422920e-01,-2.16696501e-01],[-9.46442902e-01,-1.72719657e-01,-7.09192902e-02],[2.64591992e-01,-8.10686886e-01,-1.54766694e-01]],[[-3.24388087e-01,5.79642177e-01,-4.67123777e-01],[-1.07891046e-01,-2.44596988e-01,1.57956779e-01],[-2.20612422e-01,-2.13517085e-01,-3.20238292e-01]],[[-9.64445651e-01,-1.20849021e-01,1.56455650e-03],[-1.30645990e-01,6.69570148e-01,-6.51030123e-01],[5.37831485e-01,-6.19854629e-01,1.85084790e-01]],[[-1.43823862e-01,1.65747836e-01,-1.61478564e-01],[-3.19476753e-01,-7.80723691e-01,-1.21201470e-01],[-5.70677459e-01,-1.21573083e-01,-1.13211147e-01]],[[5.01685264e-03,2.95480132e-01,1.05096839e-01],[-1.56887203e-01,1.89101800e-01,-1.57324433e-01],[-1.55015782e-01,-1.32298529e+00,2.17838734e-02]],[[2.60752827e-01,-4.68987189e-02,-8.10197055e-01],[2.02068448e-01,-2.16583431e-01,-4.85320091e-02],[-6.77386165e-01,-8.53910625e-01,-1.45295754e-01]],[[2.03023016e-01,4.03431535e-01,1.74661160e-01],[7.34986141e-02,5.23140252e-01,-1.88684314e-01],[-2.46183217e-01,1.25150293e-01,-6.72015846e-01]],[[-4.56832707e-01,6.15656853e-01,-3.40335041e-01],[3.78120691e-01,-2.81821966e-01,1.14317894e-01],[1.81901127e-01,-1.07973874e+00,1.35504097e-01]],[[-6.14193566e-02,6.68881908e-02,5.14546990e-01],[4.66426602e-04,1.60499454e-01,-1.64514676e-01],[-5.49077570e-01,-2.21153855e-01,-2.06898242e-01]],[[-6.86170101e-01,4.32465822e-01,-1.11166604e-01],[-7.34112561e-01,-1.99354261e-01,5.78535721e-02],[-2.98708409e-01,8.12952161e-01,1.56155556e-01]],[[-2.51684308e-01,3.48431766e-01,8.71970691e-03],[4.68317389e-01,4.31797691e-02,-7.93285966e-01],[7.55137920e-01,-1.18473150e-01,3.27249579e-02]],[[4.36939597e-01,-7.91286945e-01,-2.43931592e-01],[1.65658206e-01,-6.97042346e-01,2.71622270e-01],[-2.37567097e-01,-1.26519346e+00,-2.83838630e-01]],[[-1.04975134e-01,1.98124573e-01,2.91750163e-01],[7.16502488e-01,-3.91010374e-01,1.03859341e+00],[3.27778906e-01,1.14475928e-01,-3.51994038e-01]],[[6.83841705e-01,-2.78333604e-01,-4.32323944e-03],[4.24832523e-01,-2.22974181e-01,3.13899547e-01],[-3.95131946e-01,-7.21665502e-01,1.73099898e-02]],[[-5.86929500e-01,-5.16231418e-01,1.15484722e-01],[-7.85032660e-02,-3.03382128e-01,1.50397032e-01],[-4.56789076e-01,-4.80770856e-01,-8.72620288e-03]],[[-1.05399294e-02,-3.08629304e-01,1.03269696e-01],[1.66649580e-01,-1.01891232e+00,-2.22884163e-01],[-9.10506070e-01,5.28563485e-02,-2.75705278e-01]],[[-1.68899298e-01,7.26245701e-01,6.33761287e-01],[1.40942782e-01,6.49603382e-02,5.00608325e-01],[-9.87944305e-01,-9.51109469e-01,2.67055959e-01]],[[-2.39557460e-01,7.91392922e-01,8.80902350e-01],[1.57557234e-01,4.60374281e-02,3.95748019e-01],[-2.13652074e-01,-3.24778169e-01,-1.04331899e+00]],[[-4.68337201e-02,-1.23855911e-01,2.09815323e-01],[7.81466126e-01,6.66587114e-01,-3.79007906e-02],[2.58681178e-02,-2.22175568e-01,-3.31501245e-01]],[[-5.64207017e-01,1.03365374e+00,-3.28426361e-01],[6.19859338e-01,2.90270835e-01,-2.13021889e-01],[3.64898860e-01,-4.83357102e-01,-4.11944836e-01]],[[-2.46348709e-01,1.98922276e-01,2.48900801e-01],[1.48918837e-01,1.97709575e-01,3.61489207e-01],[7.00781822e-01,5.66829070e-02,-1.18421161e+00]],[[-5.54653287e-01,-9.70025808e-02,5.35640776e-01],[-2.52369493e-01,4.63601589e-01,4.37626511e-01],[4.38860297e-01,3.99093211e-01,6.70840323e-01]],[[4.87117708e-01,3.64083946e-01,4.94525842e-02],[2.54336566e-01,3.67552459e-01,2.58751571e-01],[-2.89793342e-01,9.28018764e-02,1.23782761e-01]],[[5.35444915e-01,7.11382106e-02,-2.36442372e-01],[1.31371185e-01,-7.58793950e-01,-2.28295237e-01],[-2.20699996e-01,-1.32170513e-01,2.46920109e-01]],[[3.96425903e-01,-1.69327572e-01,-2.66369492e-01],[2.28983253e-01,1.62345678e-01,4.17187423e-01],[5.49768090e-01,2.16670483e-01,-5.53557158e-01]],[[7.89251208e-01,-1.49006516e-01,-2.17337400e-01],[-4.61914629e-01,4.16063339e-01,6.50249541e-01],[-4.36745957e-02,-2.47759789e-01,1.72202691e-01]],[[-6.70575142e-01,2.31256057e-03,2.49508962e-01],[2.95071483e-01,-7.63702452e-01,9.31310534e-01],[9.98709202e-01,4.71054971e-01,8.97876084e-01]],[[3.07806462e-01,-3.23069870e-01,5.56285322e-01],[-6.00514889e-01,3.14922154e-01,-2.63162851e-01],[-1.08329403e+00,-1.35364786e-01,-2.02794969e-01]],[[5.16909420e-01,8.24664161e-02,-1.01282299e-01],[4.03184369e-02,-5.26105106e-01,4.83405739e-01],[-1.80928901e-01,-8.30186069e-01,-5.74926078e-01]],[[1.14481509e-01,2.46351704e-01,1.21981345e-01],[1.47746295e-01,1.18339956e-01,-1.79916412e-01],[1.75963491e-01,-1.74334854e-01,1.72025114e-01]],[[-1.07688451e+00,-4.05897014e-02,2.59468611e-02],[3.66882056e-01,5.38901985e-02,-3.61376524e-01],[7.82786012e-02,-3.83639157e-01,1.80395424e-01]],[[-4.30835992e-01,7.44114518e-02,1.70682475e-01],[3.68657172e-01,1.02666914e-01,8.59096467e-01],[-8.84569705e-01,1.98328093e-01,-7.83091247e-01]],[[-7.17895627e-01,-3.18488866e-01,-5.54758847e-01],[-1.26596844e+00,-4.70715255e-01,-4.26340967e-01],[-4.68993068e-01,-6.51379302e-02,2.84544468e-01]]],[[[-1.16298065e-01,-2.62330294e-01,4.86777946e-02],[1.49143348e-02,2.57524699e-02,-1.26109385e+00],[-2.49542177e-01,-7.71969855e-02,-1.90900266e-01]],[[2.92840481e-01,-6.30598247e-01,1.87839195e-01],[-3.96523863e-01,-9.77858484e-01,-1.57075599e-01],[-1.64985389e-01,7.52997994e-01,3.62935930e-01]],[[5.64386904e-01,2.33152106e-01,6.51750207e-01],[-3.85986775e-01,2.58801222e-01,-3.16199988e-01],[3.66414994e-01,-1.47141576e-01,-4.00249809e-01]],[[-4.25521463e-01,1.01521038e-01,-2.65892409e-02],[-2.90801466e-01,-1.94635078e-01,-2.41996035e-01],[-5.33455610e-02,2.07845390e-01,-1.60915518e+00]],[[-4.09172714e-01,-1.19613916e-01,9.06357408e-01],[-1.29835740e-01,4.94773716e-01,2.01858237e-01],[1.02065064e-01,9.40349221e-01,-1.97836101e-01]],[[-3.86986941e-01,1.31927997e-01,-1.18895543e+00],[-5.14373124e-01,1.46826748e-02,-8.01788643e-02],[1.56384975e-01,-7.01780319e-01,5.00911415e-01]],[[1.45232856e-01,-3.08318377e-01,-8.12851638e-02],[3.85122336e-02,2.32658446e-01,6.23844206e-01],[7.18783855e-01,1.73630595e-01,-7.37076759e-01]],[[-8.85300815e-01,4.35131162e-01,2.32011288e-01],[2.72017390e-01,3.27609926e-01,1.63206875e-01],[-1.28675342e-01,4.54663113e-02,-2.82384992e-01]],[[2.96319097e-01,-2.74884939e-01,-6.25892818e-01],[-7.53902137e-01,-2.37262070e-01,5.41113079e-01],[3.98681790e-01,1.02520871e+00,-4.45449263e-01]],[[-3.99494499e-01,3.32800537e-01,-1.04327887e-01],[-7.55085588e-01,3.94337237e-01,3.26796025e-02],[-4.66992080e-01,-6.66808903e-01,-3.23804915e-01]],[[4.19009328e-01,4.68228087e-02,-3.28134745e-01],[-5.00794277e-02,-7.13321745e-01,2.03523532e-01],[4.70150739e-01,8.16423260e-03,-1.41291440e-01]],[[1.92055926e-01,-2.54374240e-02,-2.67726868e-01],[-2.81706661e-01,4.19260323e-01,5.39354980e-01],[8.67488682e-02,-4.97133523e-01,4.54846591e-01]],[[6.98154271e-01,-5.71794510e-01,-5.93677282e-01],[-7.22203925e-02,6.14583015e-01,-4.78191048e-01],[1.25423804e-01,1.97403505e-01,3.91612500e-01]],[[-2.69902408e-01,3.46696079e-02,-1.97817869e-02],[-3.84568870e-01,4.06018078e-01,5.28572053e-02],[-1.89459935e-01,-1.58416495e-01,-7.09459007e-01]],[[-3.48269492e-01,-3.97973895e-01,-1.24968514e-01],[-3.82894248e-01,3.66098315e-01,7.28170620e-04],[-2.47641683e-01,-3.91793281e-01,1.23226270e-01]],[[6.75242767e-02,6.04584813e-01,1.08316219e+00],[3.06277901e-01,7.61371970e-01,4.05771583e-01],[-2.94962168e-01,5.80759704e-01,3.72763693e-01]],[[-5.94146252e-01,-1.35968328e-01,1.14221685e-01],[6.11888289e-01,-1.10815954e+00,2.89640009e-01],[-1.16985250e+00,8.08144927e-01,-1.52185947e-01]],[[-3.39127690e-01,6.17046766e-02,3.37796628e-01],[-1.28182247e-01,3.12483847e-01,-2.97293305e-01],[-2.60143548e-01,-1.43098414e-01,-2.12839052e-01]],[[1.36009967e-02,2.68487215e-01,6.94424272e-01],[1.16116934e-01,-2.48620957e-01,-1.22519895e-01],[3.05030644e-01,-7.16708422e-01,-9.62913036e-01]],[[-1.02181695e-01,4.56643671e-01,1.10502481e+00],[-5.27364202e-02,2.23531857e-01,8.08217049e-01],[-4.37285751e-01,-8.37615058e-02,4.02952760e-01]],[[8.58935788e-02,-1.69839710e-01,-1.32479072e+00],[-8.61281380e-02,3.79431725e-01,-4.76731092e-01],[1.21656775e-01,-5.78228116e-01,-9.63332117e-01]],[[-3.79960477e-01,2.26491824e-01,-4.30925816e-01],[-2.31662765e-01,-1.65941343e-01,3.32045019e-01],[-3.90682697e-01,-6.77576959e-01,2.09828615e-01]],[[-9.76396948e-02,-5.37823103e-02,-3.08407694e-01],[-7.16354370e-01,-3.61158729e-01,8.56391042e-02],[2.02356786e-01,-3.62225138e-02,2.56462187e-01]],[[2.75095314e-01,-1.93476215e-01,4.59197015e-01],[-5.96767187e-01,5.90304518e-03,5.25598168e-01],[-9.65925992e-01,-4.29468542e-01,5.15271008e-01]],[[-7.62636721e-01,5.19387245e-01,1.10047065e-01],[-9.03037369e-01,6.73087478e-01,6.80244386e-01],[-5.01731932e-01,-2.39549920e-01,-4.56664711e-01]],[[1.87649220e-01,-1.91829249e-01,-1.07020698e-01],[-1.16117112e-01,-4.86766756e-01,-8.08678642e-02],[1.71490967e-01,-3.50023389e-01,-5.19887805e-01]],[[-1.39212176e-01,8.65827560e-01,5.49739957e-01],[-5.76239288e-01,6.64826751e-01,1.26107311e+00],[5.42374909e-01,1.27865374e-01,-5.94250346e-03]],[[-2.44636402e-01,-2.54464477e-01,9.07034278e-01],[3.84666145e-01,1.32508129e-01,-4.30910029e-02],[5.27348220e-01,-8.61252367e-01,-9.27101433e-01]],[[-2.48186305e-01,-4.94526736e-02,-4.51716512e-01],[-1.01696206e-02,-3.61455292e-01,1.44247845e-01],[1.68057367e-01,3.03692788e-01,-4.32835333e-02]],[[2.62326062e-01,9.48619172e-02,-9.04984176e-01],[-7.11958855e-02,-1.65671900e-01,7.14584887e-01],[-4.34029818e-01,5.42513095e-02,-3.93358245e-02]],[[6.39120817e-01,4.01315033e-01,3.42852533e-01],[-2.63326675e-01,3.26872289e-01,4.94596623e-02],[-2.78643444e-02,-2.86141664e-01,9.46093276e-02]],[[-1.43188596e-01,5.66386461e-01,4.96083617e-01],[-9.42659974e-02,5.92777394e-02,9.80881453e-01],[-8.83524179e-01,4.88062054e-01,-1.40060231e-01]],[[-1.44532964e-01,-1.93008166e-02,-9.08018589e-01],[-2.84942567e-01,-8.07344168e-02,-2.51709640e-01],[9.93690118e-02,-9.43048954e-01,-1.95210680e-01]],[[6.99632287e-01,2.46605217e-01,4.60482359e-01],[-6.26216531e-01,-9.95303020e-02,2.86247969e-01],[-9.54604074e-02,4.12619650e-01,2.21797630e-01]],[[-2.62079984e-01,-5.80070734e-01,1.92321781e-02],[-3.79028797e-01,9.90826339e-02,-4.13772076e-01],[2.00004146e-01,3.23870510e-01,6.61588490e-01]],[[-2.97681838e-01,4.84551132e-01,2.61634916e-01],[-5.48691750e-02,-3.67253244e-01,-7.68065512e-01],[1.00159071e-01,-4.23069000e-02,-1.50917813e-01]],[[-6.06929511e-02,4.94374245e-01,4.64837223e-01],[2.35765517e-01,2.89828658e-01,-1.56310961e-01],[-3.00616354e-01,-2.46767953e-01,-3.24456960e-01]],[[-6.33313537e-01,-1.24201797e-01,1.72439262e-01],[-3.97266567e-01,7.06372708e-02,2.93946356e-01],[-8.88308108e-01,4.87261027e-01,8.17164898e-01]],[[3.13550800e-01,2.63765484e-01,1.35302320e-01],[-4.23808277e-01,-4.95367765e-01,3.46987605e-01],[6.85174242e-02,-3.81737463e-02,-1.50665209e-01]],[[-2.79785246e-01,1.80914849e-01,-7.09361434e-01],[2.23596767e-01,1.54266525e-02,-1.25973150e-01],[-4.71706808e-01,6.78808212e-01,3.80926281e-02]]]])
net.state_dict()['block1.6.bias'][:] = torch.tensor([-9.950513e-05,-0.0003944274,8.673868e-05,8.432643e-05,0.0004921221,-0.00058579736,2.1046959e-05,0.00021491172,-2.3213957e-05,2.8084847e-05,-0.0001446965,-2.645023e-05,-0.0005443549,-0.000118316006,0.00038215425,0.00028019428,-0.000100807985,0.0005600134,0.00047127233,0.0002552884,0.00025625696,0.0003001347,-5.664701e-05,-0.00047876273,-0.00032685144,-0.00027726247,0.0005082503,-0.00014281362,-0.00030498306,0.0003884448,-0.0005989432,-4.5458446e-06,-0.0009922237,-0.00077494397,9.403095e-05,6.0729995e-05,0.00015117505,7.551472e-05,-0.00037766778,0.0005782582,0.00039393787,-0.00016131834,0.00021938737,-0.0002844936,-0.00011185264,-0.0005131132,0.0003632541,-4.8535374e-05,0.00033305874,-0.00039456406,0.0004429088,-7.4368116e-05,0.0005409131,0.0001505419,-0.00056110317,5.8240243e-05,0.00079156214,1.0928523e-05,6.440164e-05,0.0001986773,0.0001659338,-0.00076487823,-0.00029387936,-0.001249182,0.0003709047,-0.00037020043,0.00030817353,0.00023786994,-0.00025879964,-0.00020177959,0.0005245287,-0.0004981525,-0.00037062395,-6.674978e-06,-9.1662536e-05,-0.00042887108,7.039496e-05,-0.0004677994,9.46826e-05,8.012727e-05,0.00039621262,-0.00021248421,-0.0006284495,-0.0007950714,0.00021042593,-0.000533769,-0.000120653305,0.00050591864,9.411686e-05,-0.00044069387,0.00032905932,-3.59001e-05,-0.00017898504,0.00013598998,0.0006292561,-0.00033610468,-0.00019627389,0.0005874822,0.00014913603,-0.0005516043,-0.00044757177,0.0004149944,-0.00020170516,-0.000257508,-0.0004817936,4.539096e-05,0.00038480206,6.9745e-06,-0.0006344289,-0.0002955554,-0.00055632665,-0.0008078078,0.00022895937,-0.00025945896,-0.00026454928,0.0005035668,-0.0001452374,-6.791302e-05,-0.00013941166,-0.00011534637])
net.state_dict()['block1.7.weight'][:] = torch.tensor([0.676863,0.39077646,0.5413226,0.4826824,0.59596205,0.5601587,0.48499405,0.5825167,0.46886125,0.59910613,0.4775455,0.44505087,0.61094934,0.49780166,0.51941127,0.48175,0.5622159,0.6105998,0.5025655,0.49458358,0.42890063,0.4766126,0.63958365,0.5284575,0.37355563,0.38950473,0.39413604,0.44492823,0.5873791,0.57042897,0.520487,0.60505337,0.45769715,0.47004345,0.45237297,0.4776325,0.43567356,0.5738726,0.44276378,0.44090846,0.41075453,0.49459648,0.50817424,0.4369687,0.4643959,0.53834456,0.5731486,0.65220344,0.5967,0.4694494,0.4666988,0.59508747,0.3925486,0.434204,0.5460448,0.5392599,0.48004776,0.55088145,0.49323338,0.55303025,0.46354,0.5148591,0.5462284,0.46285075,0.5171936,0.5980142,0.45123962,0.5790894,0.4716053,0.5404135,0.64017826,0.60807717,0.6125974,0.458927,0.49892828,0.5025783,0.5167483,0.42705503,0.44739586,0.5986405,0.5484822,0.57612014,0.4634347,0.51867443,0.42825416,0.55590653,0.43608245,0.46115872,0.43923932,0.4298969,0.4710695,0.67820424,0.5022471,0.46696422,0.3921271,0.49689126,0.4831422,0.35613126,0.6926863,0.45880774,0.6401721,0.40613782,0.44986507,0.4216631,0.55722344,0.47068912,0.5492031,0.66457415,0.4991016,0.4621083,0.5021189,0.56645614,0.53806764,0.48321038,0.4040958,0.56328493,0.47202468,0.5092041,0.6173213,0.55924135])
net.state_dict()['block1.7.bias'][:] = torch.tensor([-0.14086644,0.024617413,-0.21735685,-0.19796762,0.059175577,-0.16280903,0.036362782,-0.13861308,0.011117379,0.23026107,0.06255633,0.07440303,-0.098727085,-0.04982303,-0.17964564,-0.058094025,-0.11779226,0.08340467,-0.066360235,-0.055917654,0.15264736,-0.070212275,0.018508324,-0.034951143,0.09050129,0.036901347,0.042437192,0.04819968,-0.21313868,0.031260476,0.016062291,0.030573579,-0.031217087,-0.1347643,0.0153000355,-0.006779925,0.20812652,0.0069636786,-0.0058293873,0.015749143,-0.06340223,0.06688736,-0.09742541,0.056795545,-0.05266992,-0.07331972,0.049995054,-0.35281473,-0.12555541,-0.0533468,-0.14322522,0.08714459,-0.106657155,0.0732562,-0.15275937,-0.12371146,-0.15942626,-0.2551834,0.008842552,-0.17943892,0.07888637,-0.17350343,-0.06349231,0.0010016462,-0.20976925,-0.064227134,-0.12523524,0.07147406,-0.056316517,-0.09233167,-0.2079496,0.01370058,-0.14265205,-0.062668435,-0.10993068,-0.18882596,-0.121648915,0.06864684,-0.07105099,-0.25167224,-0.049331404,-0.11989216,-0.111453064,-0.10754542,-0.11918375,-0.09655019,0.025110181,-0.14444722,0.082626715,0.011046554,-0.01054774,-0.24360718,-0.030958321,-0.028822098,0.085604444,0.03995508,-0.14004683,0.11380666,-0.27096766,-0.004390439,-0.15001214,-0.012307962,0.017963814,-0.003138124,-0.08361437,-0.10922725,-0.07557496,-0.055539902,-0.012724936,0.031237155,-0.11492527,-0.058800295,-0.12893432,-0.038456254,0.043471903,-0.15025206,0.057732206,-0.11117261,-0.14803673,-0.20523784])
net.state_dict()['block1.7.running_mean'][:] = torch.tensor([-4.9095483,-0.6771935,-1.2756073,-2.4486742,-0.5794817,1.859172,-4.754873,-5.4958615,8.73597,-4.400974,-0.32603812,-1.8574846,-3.6061215,-3.916035,-4.195147,-5.0459967,-2.3104732,-4.516756,-2.8331668,12.612557,-1.4343922,-3.756725,-1.8796922,-6.9631624,4.8786836,1.112347,2.9725947,-1.7640743,-1.2355169,-1.0325948,-7.6082807,6.5890746,-0.25643978,-3.526386,1.5545274,-2.7789521,2.4861443,-5.4692793,-5.6713195,-0.9182076,-5.3906493,-1.1323677,-4.0529275,4.019969,-4.3240275,-1.783875,14.171846,2.6033645,-2.6741111,1.6007385,0.20178029,-5.3416367,-5.3293257,-3.6623206,-1.8674545,4.169942,10.495603,3.5863292,-7.8326735,-3.8738112,-2.9189248,-9.882238,-3.800293,-0.5004075,-3.9029536,-4.2621617,-1.1699225,9.244823,1.0617722,0.20429555,-1.3281481,-9.59799,-1.9112163,6.1223426,0.95182216,1.3783981,-2.5905857,0.020971693,-3.7532716,-5.8597603,-2.8599167,-4.722153,0.81392723,7.260582,-4.936862,-2.535871,-2.7467694,-0.36303115,-3.3198628,2.8044527,-1.7994411,-3.83617,-3.426629,-3.8903134,-0.2190985,-1.5318894,0.107873164,-1.1936448,-0.24137317,2.751333,-6.9225235,3.2987735,-5.107737,2.7362413,-8.063937,8.239248,-4.1632767,-6.85693,-3.3249466,-1.7001839,4.9765377,-3.104667,0.35564205,1.3985476,-3.3609486,-3.2849982,3.0906367,-1.2948807,-6.5328736,-3.805673])
net.state_dict()['block1.7.running_var'][:] = torch.tensor([44.43992,31.674618,33.524036,43.914978,48.718872,43.71157,53.253307,49.84346,44.938694,62.941372,35.02778,34.565746,48.992294,31.418945,40.93531,36.896317,45.41931,61.517498,38.104294,59.30369,33.614586,30.29677,58.63935,48.090008,28.183294,31.997637,35.64296,31.694357,31.530151,44.65488,37.20352,72.95029,41.325253,25.594286,36.603027,49.47201,41.95957,42.051785,29.253992,32.78453,28.451155,59.298576,33.5256,39.87552,33.028786,37.42591,84.17599,33.230286,44.581604,45.257786,35.063313,49.18647,29.671425,39.250328,37.20846,34.38232,41.26774,33.219578,32.50324,33.422573,29.422258,49.2099,41.350517,34.074898,27.535679,48.13691,38.023087,68.89437,29.39215,36.350555,36.95938,61.308968,40.043266,56.478985,34.938786,37.22063,30.366896,35.951065,41.8197,39.112156,41.820114,44.866665,27.564299,70.5712,25.36901,33.4384,31.156393,37.38635,36.854313,40.17416,35.15418,43.677643,49.724632,30.864393,30.158272,33.716503,33.517406,28.996323,41.17891,46.030254,42.55837,33.664425,41.56158,31.48571,45.203896,39.86972,39.75111,48.191994,37.94866,30.855106,38.633293,35.39128,28.899342,44.81549,28.968792,39.21333,35.23545,32.760532,38.485516,30.142694])
net.state_dict()['block1.7.num_batches_tracked'] = torch.tensor(305600)
net.state_dict()['block2.0.weight'][:] = torch.tensor([[-0.77538306,0.7714976,0.9091454,-0.8437174,0.45769346,-0.03069505,0.10429389,-0.06845915,0.6238458,-0.13062018,-0.6459022,0.22240031,-0.24261636,0.13253948,-0.09267317,-0.26195148,0.43935457,0.2318811,-0.57053775,-0.02987794,-0.37931538,0.71323365,0.04179955,-0.0641792,0.6328503,-0.9362037,-0.5208239,0.50293386,0.65936583,-0.07178584,0.6953577,-1.1836295,0.42748305,-0.0634667,0.49551263,0.22578973,-0.80096227,0.643016,0.5539439,-0.7043457,-0.61609536,0.14456883,0.25269854,0.72292143,-0.64979416,-0.12123191,0.37511832,-0.5411057,-0.07002947,0.45216003,-0.00339553,0.6439968,-0.04902892,-0.09527127,-0.40747994,0.06877119,0.19851238,-0.06676397,-0.1332809,-0.04121839,0.19690564,0.00418031,0.38985106,0.14285912,0.09552238,-1.1378956,0.36178648,0.14765131,-0.88866895,-0.1219797,-0.9309991,0.35201082,0.16103034,-0.52013147,0.5458687,-0.8452939,-0.18851723,0.7760482,-0.13408439,-0.06108255,0.43152902,0.25582397,0.6306102,0.0160827,-0.15963148,-0.06308492,0.2594664,0.49560845,0.27366233,0.7071819,0.25461355,0.05584478,0.2403031,0.1850222,0.38618392,0.22782263,0.4297569,0.18447648,0.43047476,-0.85376716,-0.12916282,-0.01419965,0.1849008,0.33840114,-0.07104691,-0.5162219,-0.2998929,0.27357036,-0.12023995,-0.23304123,-0.49419153,-0.4293808,-0.03572079,-0.00269142,-0.31568766,-0.17946775,0.02403337,-0.2573757,-0.40970615,0.01258201,0.10864476,0.31487113,-0.37288466,0.02697217,-0.36713672,-0.34040087,-0.6355179,0.16640607,-0.4034727,-0.14583935,-0.79430425,0.6092067,-0.17595531,-0.45617715,-0.06781361,-0.3288096,0.06040258,-0.08771536,0.32003823,-0.31122997,0.06408612,-0.5150653,0.12774831,0.2443237,-0.42689222,0.3420232,0.44704738,-0.55261326,-0.24386129,-0.55242455,0.44125143,-0.10098635,0.2938397,0.28432256,-0.60088027,-0.18517147,-0.6044737,0.23916307,-0.27659473,0.45229134,0.2298904,-0.07725033,-0.07336733,0.973484,-0.7028383,0.19350456,-0.8992649,0.33592048,0.56104296,0.19427498,-0.7496489,-0.09803274,-0.01404842,0.17605516,-0.29223636,-0.35624018,-0.22423016,0.49568886,-0.12540816,-0.28999716,-0.04974017,0.41319153,-0.5574887,0.06330273,0.4394689,-0.24739505,-0.24250557,0.34169927,0.12343239,0.22807634,-0.5208895,0.3383372,0.69720155,0.30079,-0.29483512,-0.6618724,-0.36248,0.6850547,-0.243557,0.47246826,0.2970403,-0.30209514,-0.97775924,0.5758792,-0.20148562,0.05586563,-0.75823313,0.3828321,-0.45218942,0.36325645,-0.54869556,-0.37007815,-0.20092689,-0.37289053,-0.47863397,0.36371306,0.4932772,0.54489046,0.13799356,0.5998031,0.18965559,0.28219172,0.51263684,0.01782273,-0.5809285,0.39687404,0.54039305,-0.00352444,-0.42046928,-0.25035077,-0.12921983,-0.45706832,0.3899544,-0.27792707,-0.09030915,0.3765478,0.17203367,-0.7839682,0.29052532,-0.07393084],[-0.24621664,0.540433,-0.40369338,0.5139055,-0.17620529,-0.08228589,-0.3236971,-1.3759022,0.3859865,0.35133043,-0.37536976,-0.14324494,-0.39774,0.40859142,-0.20592386,0.37334776,-0.52673554,0.28305197,0.54980874,-0.2586956,0.36393106,-0.59940624,-0.48532447,0.18086708,0.15414573,-0.35441607,0.28802425,0.15400779,-0.17277567,0.42723542,0.9710618,0.05229152,-0.56925946,0.5446822,0.36091754,0.00272085,0.13061629,0.17771162,-0.13831788,0.4895569,-0.17113157,-0.2405251,0.10223562,0.05243981,-0.23293869,-0.2654437,-0.07125929,0.42927748,-0.5464007,-0.4091557,0.21128228,0.154085,-0.2158768,-0.04874679,0.06770154,-0.5872217,0.7924906,-0.17972678,0.47800818,-0.1535861,0.26287714,0.06153588,0.24224082,0.3248157,0.01932045,-0.41384432,-0.02112215,0.6394346,-0.16913593,0.30250564,-0.41849002,0.5023578,0.35580915,-1.1501327,0.2208386,1.1741208,-0.48988685,-0.2085368,0.04406178,-0.3468514,0.2624227,-0.9592456,-0.01869651,-0.22954744,0.35396853,0.37698576,0.27883843,-0.52433026,-0.02125071,0.28701422,-0.04503731,0.14746822,0.11130907,-0.10059142,-0.25139576,0.67483515,0.28305435,-0.362744,-0.71243995,0.3152524,0.20516615,-0.3504651,-0.18831792,0.05119906,0.5051619,-0.13194254,-0.07696543,0.15384291,0.31175563,-0.8707897,0.11864311,-0.9185299,-0.5250081,0.27215925,-0.8331821,-0.6287804,0.34382355,-0.8990389,-0.29441848,-0.11825271,0.14878221,0.44276255,0.09373741,0.21501614,-0.22083494,0.85545737,-1.4134899,0.09111573,-0.9492083,-0.3611324,0.17429888,-0.8685013,0.15561464,0.50287586,-0.791484,-0.22853844,0.29627952,0.4633886,-0.41996807,0.18252721,0.17903647,-0.14449488,0.7308775,-1.4061885,0.5349823,-0.47698686,0.04373758,-0.1116097,-0.2649864,0.57444143,-0.4526126,-0.91911304,-0.40180394,-0.0553402,-0.7489939,-0.67265415,-0.6849432,0.2381136,-0.1655641,-0.16285534,-0.19780143,-0.15562412,0.21481708,-0.20790187,0.10768402,-0.91263855,-0.50369775,-0.0266672,0.22596262,-0.23903455,-0.15354815,-0.0762505,0.13450775,0.05950417,0.13652657,-0.3281492,0.3452058,0.67671686,-0.32745105,0.43058595,-0.17760839,0.8760941,-0.11721291,-0.23521699,-0.26330018,0.5890487,-0.14640817,0.19849907,0.12996857,0.03973864,0.4850457,0.6204531,-0.27966702,-0.38970894,-0.10614198,-0.54975915,-0.5077546,0.33854035,-0.40721607,-0.07736047,0.48141044,-0.3239327,0.02064244,-0.43542218,0.15894936,0.00157622,-0.42846942,-0.13495207,0.01617564,-0.7272858,0.44320822,0.04378198,0.11268284,-0.09198842,0.19575822,-0.22831586,0.4667099,0.21501733,0.08302552,-0.16515082,0.16424808,-0.5931448,0.15752456,0.48753682,-0.77892023,0.58825386,0.14469291,0.12436399,-0.11505017,0.02732251,0.05422582,-0.17849587,-0.9355839,0.21833593,-0.23956569,-0.48370424,-0.08478215,-0.27434254,-0.14836024,0.5678855],[-0.05235564,-0.09747237,0.03982945,0.16100648,0.53383124,-1.1240773,-0.22003259,-0.11550332,0.30629712,-0.3168276,-0.64729697,0.20522231,0.25592652,-0.95231825,0.03258483,-0.59448934,-0.3447999,0.14041324,-0.19396843,-0.12282809,0.3036663,-0.02564516,-0.32864854,0.35811597,-0.01261267,0.01455186,-1.1973886,0.26768044,0.60598695,0.855325,0.08798429,-0.23886816,0.26427674,0.70337254,0.3343492,0.56225604,-0.19580127,-0.26532388,-0.1313215,-0.41335842,-0.12757584,0.11649043,0.30546957,-0.46182173,-0.19461186,-0.93881273,0.44937894,-0.5454481,0.3725103,-0.49036995,0.3775649,-0.02609619,-0.20102158,0.2384803,0.7082656,-0.609905,-0.5258643,-0.63184506,0.47921836,-0.41586524,-0.02900593,0.66765726,-0.52696335,0.26200768,-0.24540938,-0.15979923,-0.25441283,0.17187354,0.19215687,0.11482777,0.6328837,0.3202841,-0.01327871,-0.31131628,-0.09559202,0.01511725,0.22515982,-0.37786502,-0.3379672,-0.35526288,0.5705191,0.2649168,-0.29317743,-0.0581411,0.5309824,-0.3165042,0.08007525,0.07035176,-0.49669647,-0.32321358,-0.6684904,0.12815267,-0.1657408,0.37737936,-0.32039636,-0.01303795,-0.03852623,0.19890757,-0.12970303,0.6149126,0.04163941,0.20223856,0.5689611,-0.5673915,0.14802341,0.2672116,-0.25791067,-0.46425533,0.37173888,0.17452288,0.33400467,-0.57945776,0.2683415,-0.9522313,-0.6788228,-0.17541933,-0.05846283,-0.6877364,0.61123294,-0.08440298,0.6055675,-0.4388084,-0.89712334,-0.40365744,-0.37430438,-0.01300822,-0.7412749,0.5583491,0.48309204,-0.16551262,0.5616616,-0.12341104,-0.11462,0.5448311,0.31644106,-0.1396213,-0.19877233,0.21383974,-0.04652276,-0.6304138,-0.55984163,-0.09305491,-0.4971331,0.4541328,-0.84137654,0.30592814,-0.46547303,0.29983073,-0.05947105,-0.00795154,-0.48407865,0.01337676,-0.11754348,0.40110517,-0.26881674,-0.06613848,0.4762001,-0.12803236,-0.12666163,-0.35916346,-0.11897996,0.1920545,0.3047572,-1.0200087,-0.6104934,-0.04473485,-0.013575,0.5756433,-0.3064151,0.01528296,-0.30217525,0.27670968,0.68322515,0.67649025,-0.00950498,0.14747074,-0.09434793,0.15569471,-0.26669723,0.08589243,-0.3883979,-0.6836261,0.40370825,-0.644803,-0.5267363,0.1010541,-0.3619139,-0.06686346,-0.1432834,-0.6276509,-0.3866624,0.32625097,0.43024397,-0.30527857,0.4487196,-0.05184251,-0.15407635,0.46168277,0.47924644,-1.144126,0.65394944,0.10816485,0.04344716,-0.28838167,0.23831859,-0.42631397,-0.4493604,0.685714,-0.638193,0.3771143,-0.89926106,0.32828557,1.1016412,0.68217134,-0.61498266,-0.59633195,-1.0472251,-0.6976786,-0.44617438,0.39599103,0.11549624,-0.24299258,0.3412348,0.17911223,-0.21966624,0.1721396,0.56601787,0.11673372,0.29363036,0.5704913,0.565256,-0.5185903,-0.35768864,0.42780825,-0.3038425,0.02883732,0.06948794,0.09307908,0.08832939,-0.3600346],[-3.03570896e-01,4.43521375e-03,-8.20912242e-01,-5.60009897e-01,4.85757649e-01,1.16270013e-01,-5.09327531e-01,-2.23526463e-01,4.75428164e-01,6.00849211e-01,8.33509192e-02,1.85411811e-01,1.69833064e-01,-4.62048724e-02,-4.71664906e-01,9.82444286e-02,4.56235826e-01,-2.84596562e-01,3.48269641e-02,4.28831279e-01,-5.37511408e-02,-3.01951021e-01,4.72848237e-01,-1.95000052e-01,-1.85521115e-02,-4.95655276e-02,2.41187006e-01,2.28060678e-01,-3.44502211e-01,-4.05412495e-01,3.61358258e-03,-1.62363291e-01,-1.58218443e-01,4.83637929e-01,2.77016521e-01,-2.11619332e-01,4.54911947e-01,-1.93558231e-01,-3.72364730e-01,1.54648215e-01,3.14634234e-01,-2.62068927e-01,4.15574968e-01,-5.80794930e-01,4.34244156e-01,5.19920409e-01,3.54390204e-01,9.87494141e-02,6.00697935e-01,-6.79531693e-01,-6.32808685e-01,2.36392707e-01,-2.23756731e-01,-6.49154261e-02,4.56290431e-02,-6.40221357e-01,-4.59293164e-02,-5.08628786e-02,1.33320451e-01,2.02835590e-01,-1.89833760e-01,2.79148251e-01,-5.94752491e-01,-9.55741346e-01,3.28731894e-01,-1.09068938e-01,-4.99671310e-01,-9.96231586e-02,-6.94480479e-01,6.91194475e-01,-1.96839720e-01,2.25205332e-01,3.03637743e-01,-6.19578920e-02,1.41044408e-01,1.01152450e-01,1.18603513e-01,1.68877766e-01,-5.65080233e-02,-4.93246764e-01,-1.15784132e+00,8.52927938e-02,-3.32381874e-01,7.31484070e-02,3.00850216e-02,4.28723902e-01,-2.11384147e-01,-5.52693307e-01,7.47474611e-01,-1.38325751e-01,-3.76119703e-01,-2.49850824e-01,-4.69483733e-01,-6.06059849e-01,-6.48353398e-01,2.26767600e-01,-6.54930472e-02,3.85900170e-01,-6.61002612e-03,4.12740678e-01,-2.38672942e-02,1.81398734e-01,3.36667925e-01,2.06256330e-01,-4.87354964e-01,9.83588636e-01,2.28380740e-01,4.49627966e-01,-3.66254747e-01,3.32019180e-01,2.13719651e-01,2.12203085e-01,-3.81662130e-01,-2.47708216e-01,-4.32172865e-01,-6.81926131e-01,3.10892075e-01,-7.10368097e-01,-7.98769295e-01,2.11114034e-01,1.62901282e-02,1.58036232e-01,-4.22007769e-01,4.51633066e-01,-2.77229786e-01,-6.79714829e-02,-1.24665461e-02,-6.33455276e-01,3.38440269e-01,-4.62523758e-01,-3.19402642e-03,8.38052869e-01,-8.72163475e-02,6.78716749e-02,-1.10466547e-01,2.20215321e-01,-3.19779098e-01,-1.46237701e-01,-4.35921252e-01,4.13392484e-01,-1.03305608e-01,2.71668971e-01,-3.10864653e-02,6.70850635e-01,6.49265707e-01,-3.36606234e-01,-8.51283252e-01,-8.93173963e-02,-5.39224267e-01,-4.30349052e-01,4.59570050e-01,-1.57973692e-01,3.53206992e-01,7.05380738e-01,6.59863591e-01,-3.62949610e-01,1.71561375e-01,-2.10479394e-01,-4.03250940e-02,-4.10063416e-01,-9.91236985e-01,7.16672897e-01,-1.61250398e-01,-2.30943158e-01,-1.62881855e-02,8.15171897e-02,-2.35334665e-01,-6.07558548e-01,3.14154476e-01,6.45847917e-02,2.87227780e-01,-1.50134727e-01,6.86599672e-01,-5.18613338e-01,-2.03185469e-01,1.37584042e-02,3.42394114e-01,-8.45135597e-04,-8.12073112e-01,-1.48465082e-01,1.03081845e-01,-1.05672285e-01,-5.00258327e-01,-5.93122303e-01,1.47710219e-01,2.99147695e-01,6.36685342e-02,-7.67917097e-01,3.28877382e-02,4.32999104e-01,2.76301026e-01,6.37652397e-01,-3.48121822e-02,1.98669016e-01,-2.30116338e-01,4.46374476e-01,-3.46966013e-02,7.36352324e-01,-6.23881519e-02,5.58617771e-01,7.06449151e-01,7.63134509e-02,-3.78011167e-02,1.78538054e-01,2.61664093e-01,6.11572504e-01,8.55243623e-01,-3.26555490e-01,3.86239439e-01,1.26503420e+00,-7.35021293e-01,5.13246059e-01,1.89555615e-01,3.00799161e-02,6.93261802e-01,-1.28045619e-01,1.11558296e-01,-3.65430951e-01,-1.64098769e-01,-1.08112618e-01,1.48597836e-01,-8.18890393e-01,-7.65879601e-02,1.75871372e-01,2.08910238e-02,4.59712073e-02,2.70815492e-01,7.87083805e-01,5.80574386e-02,2.79172897e-01,3.09925377e-01,-4.40438092e-01,-5.82671165e-02,1.13406964e-01,1.51849166e-01,2.91800320e-01,-4.16842252e-02,1.73073947e-01,4.73655194e-01,-1.62712902e-01],[2.01752409e-01,-3.89511362e-02,-4.84222621e-01,4.21737194e-01,-6.12066627e-01,-2.96003699e-01,-1.16675608e-01,-2.67095715e-01,2.02209458e-01,-3.78893944e-03,1.69694796e-01,-5.85149646e-01,-2.92299211e-01,-1.55447567e+00,-1.09374750e+00,9.55603838e-01,-1.58747301e-01,3.80769044e-01,-8.80371749e-01,4.79114056e-01,-9.58883911e-02,-1.01300776e-01,3.55188310e-01,2.77307183e-01,-1.17740497e-01,-2.35187247e-01,2.80286103e-01,3.05920929e-01,3.05370510e-01,-5.98304570e-01,-9.67259929e-02,2.90674716e-01,2.20183253e-01,-7.09499776e-01,3.17938268e-01,-6.61071301e-01,5.59170246e-01,3.22224349e-01,-5.55651188e-01,4.88671273e-01,-4.85299170e-01,-9.38869894e-01,-2.12584376e-01,-1.41291451e+00,1.30118698e-01,3.53321970e-01,-3.69272560e-01,2.90289917e-03,2.31440231e-01,-6.01639032e-01,-3.02895218e-01,7.87627280e-01,2.84578092e-03,3.41268748e-01,-2.57685214e-01,2.36832704e-02,-3.61539125e-01,-1.15362108e+00,5.25571585e-01,-7.59031951e-01,4.00598764e-01,-4.34918404e-01,-3.53170276e-01,-2.30470672e-01,-1.00279272e+00,1.10784426e-01,2.19773073e-02,5.91856778e-01,-7.04126316e-04,2.34287396e-01,1.76994838e-02,-4.48009968e-01,1.42343670e-01,7.65456706e-02,3.99061352e-01,-1.76367253e-01,-2.86797106e-01,1.12346537e-01,-6.85085773e-01,1.91799551e-01,-5.15939713e-01,-3.33656728e-01,2.90989220e-01,3.56021561e-02,-6.21330678e-01,-1.33644074e-01,1.10764787e-01,-3.60055804e-01,-4.16594207e-01,1.92277268e-01,-5.84978163e-01,-6.81998253e-01,9.94155854e-02,1.11216024e-01,-1.19834535e-01,-5.77601433e-01,-7.97744766e-02,-9.70168561e-02,8.49990547e-02,-8.00899789e-02,-5.31382680e-01,8.03236961e-01,-9.18613002e-02,-3.54841352e-01,-6.18807137e-01,-6.81704953e-02,-2.46778965e-01,-7.37585783e-01,4.76325929e-01,-2.62019008e-01,-1.28310502e-01,6.63384914e-01,2.03144208e-01,-3.38562965e-01,-1.66872695e-01,4.73918259e-01,-7.19399750e-01,-6.78870916e-01,-3.59689981e-01,-1.89157397e-01,-2.09771506e-02,-1.28165960e+00,3.94225389e-01,-5.41949809e-01,4.80741449e-02,2.37519071e-01,-1.65924951e-02,-2.58088671e-02,1.42182549e-02,-5.29678762e-01,-2.69399077e-01,7.13275492e-01,-2.35198155e-01,1.85902372e-01,-2.01411068e-01,-8.32286254e-02,1.75786883e-01,-7.86752582e-01,2.81765312e-01,-7.69548178e-01,-5.26575148e-01,-3.41010898e-01,-8.98326263e-02,1.94700854e-03,7.28828311e-02,1.44786434e-02,3.01225353e-02,-1.02675430e-01,-2.16794580e-01,2.44498268e-01,-5.99724054e-01,2.28439435e-01,-1.46122277e-01,4.03354645e-01,-5.37471056e-01,5.50188124e-01,-1.30629689e-01,-6.43039644e-01,1.12166174e-01,-3.72205168e-01,-7.92790055e-01,-2.04110444e-01,4.35618520e-01,3.14772844e-01,2.06251785e-01,-1.57379997e+00,-2.17840299e-02,-2.97244668e-01,-4.00899380e-01,5.86707354e-01,-6.06745541e-01,-1.04646564e-01,-2.04225078e-01,1.89780876e-01,-4.68401700e-01,9.60791528e-01,-2.25355420e-02,1.25018060e+00,6.41496420e-01,2.93861151e-01,2.90504634e-01,9.12961438e-02,-1.14557460e-01,3.82329710e-02,-5.21439873e-03,8.84824097e-01,-8.86101425e-02,-7.22372830e-01,-4.62716728e-01,7.45308042e-01,-2.72904992e-01,-5.67939878e-01,4.63717133e-02,-4.51227635e-01,-6.12742640e-02,-7.06430711e-03,-1.35465369e-01,-5.07303715e-01,1.60308719e-01,-4.62402225e-01,-2.44829252e-01,3.16019088e-01,-6.07508533e-02,-9.55629826e-01,-2.68128842e-01,-6.94062840e-03,-3.75576839e-02,9.89360332e-01,-7.01867282e-01,6.84376806e-02,-2.55803019e-01,3.19286048e-01,2.63753440e-02,-1.07699156e+00,5.27858697e-02,5.05261004e-01,-3.46070081e-01,-6.98026001e-01,3.30921710e-01,-4.32335228e-01,3.06508914e-02,-2.89978147e-01,4.52108681e-03,3.57977271e-01,2.58468866e-01,2.07470953e-02,-5.07456899e-01,-3.76659274e-01,4.71169539e-02,-1.68774202e-01,-3.32685322e-01,1.54184192e-01,-5.62997997e-01,1.15431285e+00,1.93546787e-02,5.98110020e-01,-1.87253475e-01,3.89635205e-01,4.36586857e-01,-8.99791062e-01],[-2.01708347e-01,1.28882095e-01,-5.56722004e-03,-5.19841433e-01,-2.12015361e-02,1.62176177e-01,-2.26147547e-01,-1.37289017e-02,-1.17632377e+00,-1.37445509e-01,4.85855073e-01,3.71241778e-01,2.78692767e-02,3.30356658e-01,2.16899887e-01,8.04474413e-01,-7.26316124e-02,-3.42726350e-01,-7.67193884e-02,1.83580801e-01,-7.82689378e-02,-6.34719431e-01,-6.48543835e-01,1.00725286e-01,4.49537069e-01,6.67740852e-02,-2.62148768e-01,-4.55054373e-01,-4.70255643e-01,-1.53401926e-01,6.47842288e-02,2.98991084e-01,-2.46787101e-01,2.70038396e-01,-1.39766544e-01,2.69993842e-01,1.74218610e-01,9.59039569e-01,4.30758059e-01,9.64984298e-02,2.04760119e-01,6.55031860e-01,5.00308156e-01,-8.18267115e-04,2.42547646e-01,1.22183061e+00,2.37772584e-01,3.14861983e-01,1.93317309e-02,-2.40071164e-03,3.12344164e-01,3.38210702e-01,-7.88845867e-02,-5.24590835e-02,6.09653175e-01,2.34022707e-01,6.93603039e-01,-2.89778352e-01,9.45367873e-01,3.70200008e-01,3.73619139e-01,-5.45128942e-01,-3.13382268e-01,2.38238513e-01,1.95297152e-01,7.63009638e-02,2.48981953e-01,4.30767000e-01,-3.81446838e-01,1.14417365e-02,1.92044988e-01,1.37575403e-01,-3.60763699e-01,2.09117904e-01,-4.37752962e-01,1.29585847e-01,-1.88120186e-01,6.03121221e-01,-8.63372624e-01,4.65763807e-01,-8.21097661e-03,-5.65710776e-02,5.24837554e-01,2.20868081e-01,-7.08575130e-01,-2.86942035e-01,-3.96594070e-02,4.48797047e-01,-3.76202345e-01,3.08304697e-01,2.32778639e-01,-1.50439993e-01,-5.62959015e-01,1.53140292e-01,-7.05480933e-01,-3.34714837e-02,-5.75902700e-01,-9.41126719e-02,-2.27099612e-01,-1.79784268e-01,2.17301115e-01,5.30657113e-01,1.96584091e-01,-2.48060137e-01,-3.31742972e-01,3.31109941e-01,2.20289305e-01,-2.72089969e-02,3.69825006e-01,2.45893691e-02,-5.95601559e-01,3.72849733e-01,-7.81483114e-01,7.69350231e-01,-2.29618117e-01,-6.74759209e-01,1.10831954e-01,-2.74321824e-01,1.47611797e-01,-3.57799709e-01,-1.05530858e-01,-1.08232272e+00,6.28734902e-02,-3.56154710e-01,1.22872598e-01,-3.85579646e-01,-7.34123766e-01,-4.65869755e-01,-2.73498833e-01,5.71166910e-02,-4.50829357e-01,8.02733153e-02,-2.02719420e-01,2.38166675e-01,1.85356271e-02,5.01013637e-01,2.36257553e-01,2.16090493e-02,9.64514166e-02,4.04637903e-01,2.57274449e-01,1.19572533e-02,1.83867335e-01,6.85596913e-02,-1.21447548e-01,3.11490536e-01,-1.92721471e-01,1.27972156e-01,-4.64657068e-01,1.71599507e-01,-3.91213030e-01,5.29289603e-01,3.86436015e-01,-4.17781323e-01,-3.15621465e-01,6.00946844e-02,-5.55818856e-01,-7.89166331e-01,9.91332009e-02,-4.80398536e-01,4.08410102e-01,1.74620822e-01,-6.54759854e-02,-8.62674117e-02,-5.30679431e-03,4.47922200e-01,-6.43632770e-01,2.19845176e-01,-3.56342942e-01,4.69908893e-01,-5.67668676e-01,-2.72234231e-01,1.71289891e-02,7.74444222e-01,-8.24622571e-01,1.97848827e-01,-4.84105796e-02,6.67801499e-01,4.24962491e-01,-1.91754118e-01,-1.39130980e-01,-9.06522498e-02,-2.74104655e-01,4.00408626e-01,-4.21250582e-01,-9.21363756e-03,-6.79208100e-01,-1.63246274e-01,-5.04222393e-01,2.67123850e-03,-8.82827699e-01,7.18370825e-02,-1.23098023e-01,3.64173174e-01,-1.92174479e-01,1.05554938e-01,-2.70688772e-01,2.18796849e-01,-4.09098417e-01,1.09158170e+00,-6.48440778e-01,-6.36430740e-01,-1.12958997e-01,8.73225182e-02,6.08096838e-01,-1.25459209e-01,1.50736108e-01,-6.02587275e-02,-4.06748027e-01,-9.06675875e-01,-2.55874336e-01,4.00467813e-01,2.53598183e-01,2.37416863e-01,-1.66950166e+00,-2.62758791e-01,6.93244100e-01,-2.92640984e-01,2.55259275e-01,3.17022592e-01,2.07730934e-01,-2.59534150e-01,-4.58585054e-01,-3.99628013e-01,3.08259964e-01,-1.67983875e-01,-4.29618955e-01,-1.69021472e-01,-1.51114404e-01,2.59484947e-01,-4.95612741e-01,-5.72103560e-01,-5.40755130e-02,-3.34752291e-01,8.90576005e-01,-3.27059269e-01,1.22821830e-01,-1.62428737e-01,-1.14637911e+00,2.05567285e-01],[0.5239029,0.49461037,-0.5211351,0.31898674,-0.05829462,0.28277916,-0.2538492,0.28530014,0.8018474,0.06863771,0.15063128,0.51080847,-0.8207231,0.27219275,-0.28180122,0.14975396,0.2283462,-0.5929672,-0.39435226,-0.0901998,-0.19220631,0.09977993,-0.70175564,0.09480343,-0.17621997,0.13159359,-0.07554863,-0.666534,0.4768373,-0.17155902,-1.2859248,-0.0475333,0.09880363,0.024126,0.7831097,0.7350558,-0.16350572,-0.3440599,-0.17319593,-0.0188413,0.00908704,0.49160242,0.73758024,-0.30041263,0.96295226,0.19143385,-1.2344103,0.46528542,0.2665463,0.00390835,0.46891662,0.53111684,0.25849468,-0.3939573,-0.2761977,-0.22872931,-0.7480693,-0.21053216,0.04327701,0.14055702,-0.5756427,-0.7750538,0.25186628,-0.28597412,-0.17020614,-0.43195093,0.19190921,0.14065978,0.4535815,-0.07002766,-0.02384537,0.6195076,0.41538095,0.70005614,0.12705852,0.70221907,-0.7812872,-0.40478554,0.18664491,-0.51752776,-0.2654055,0.3228594,-0.3395006,-0.5397844,-0.21355252,0.6189394,-0.16226028,0.06652951,1.0242424,-0.42747408,0.01454641,0.10339943,-0.07222751,-0.5097795,-0.11529394,0.21253423,-0.42466974,-0.21454746,-0.20339273,0.17588511,-0.20392063,-0.2642655,-0.3521077,0.3335667,0.10900785,-0.2692151,-0.388329,0.44311452,0.02847337,0.10017239,-0.43737295,0.0693109,0.56740886,0.24176517,0.3902244,-0.22247453,0.00719667,-0.6713395,-0.08266044,0.36246014,0.52917796,-0.12533917,-0.6634285,0.27338088,0.21671034,0.07233836,0.03119557,-0.09611329,0.12383491,-0.40271628,-0.35359335,-0.26561692,-0.4615827,-0.19219881,-0.11161318,-0.33819845,-0.4093873,0.03706218,-0.03863492,-0.17678587,0.501436,-0.18897931,0.36250168,-0.14703889,-0.14608985,-0.49788114,-1.1089661,-0.13777076,-0.46050578,-0.40881455,0.05009868,-0.2427516,0.12453717,0.42463282,0.30639988,0.22951354,0.33365104,-0.04041401,0.26704586,-0.3785526,-0.4905794,-0.03094562,-0.79647666,0.33462107,-0.04289948,-0.21724945,0.10930322,0.23875836,0.63714,-0.24948528,-0.323062,-0.1653116,0.01173496,0.47914457,0.05546691,-0.01549481,-0.26684362,-0.05785071,0.60634327,-0.23830685,0.07059395,0.2075581,-0.00513787,0.47951597,-0.6825718,0.2104667,-0.50891834,0.5276642,-0.49916175,0.17413975,-0.19280274,-0.3034735,0.6465225,-0.3186217,0.26398245,0.04643388,-0.3364839,-0.05803116,0.6276287,-1.3538284,0.41894716,0.27790955,-0.42945248,0.78426,-0.082163,-0.13767315,-0.50894564,0.5043507,-0.27917132,0.25258926,-1.0962924,0.22381115,-0.8037265,0.12349655,-0.6921496,-0.22297677,-0.45374948,-0.42266163,0.72646695,-0.2792941,0.40291947,-0.02804641,0.9471738,-0.2053941,-0.7704106,-0.22543396,-0.47757128,-1.0376365,-0.36304173,0.05156619,0.8000055,-0.28858975,-0.19355479,0.38396102,0.6673175,-0.5247268,0.42736897,0.12595724,0.8048879,0.2175831],[5.99327147e-01,-2.36281961e-01,3.43572378e-01,-3.55993688e-01,3.12634706e-01,-2.53898561e-01,1.92971811e-01,-3.79731208e-02,-1.64106791e-03,-3.03880900e-01,-2.74953563e-02,-8.85648578e-02,4.40510422e-01,-4.26008373e-01,2.11633250e-01,9.43908617e-02,1.95135280e-01,-5.98426014e-02,-8.84318575e-02,-6.26147568e-01,-3.49068969e-01,4.67098840e-02,1.63628757e-02,-1.51669219e-01,3.74132425e-01,-1.20394520e-01,5.89243591e-01,-4.54649359e-01,3.02368719e-02,2.14282632e-01,-4.92939889e-01,4.69705671e-01,-2.67207444e-01,4.25998122e-01,3.76742423e-01,5.49632847e-01,3.92448753e-01,3.47157985e-01,-8.27191234e-01,-1.57842442e-01,1.19544063e-02,3.89515966e-01,-4.62764874e-02,1.19611971e-01,2.22588759e-02,-1.66599248e-02,-1.49425179e-01,5.53743899e-01,8.78428072e-02,-2.84504145e-01,7.65353024e-01,1.96323842e-02,2.30760634e-01,6.77037895e-01,-4.09496650e-02,2.64321387e-01,-2.48926222e-01,-1.77634463e-01,-7.11421609e-01,-3.39788139e-01,7.77955353e-02,-9.17346799e-04,3.40478510e-01,-3.98950309e-01,-1.79248840e-01,4.65093881e-01,2.48296142e-01,-5.85511625e-01,-1.04755357e-01,-6.94765806e-01,-1.02909133e-01,2.56173193e-01,4.30968761e-01,-1.07167079e-03,-9.00742710e-01,-2.55035549e-01,3.14490139e-01,-1.88404307e-01,5.42931914e-01,-2.18971204e-02,-5.31551182e-01,-1.21784985e-01,1.31099209e-01,5.11303782e-01,-1.53990351e-02,-1.52074680e-01,-4.57071275e-01,1.36869803e-01,7.54180193e-01,-3.14702898e-01,1.03073962e-01,3.62349957e-01,-5.82942307e-01,5.58160469e-02,-3.52688760e-01,1.99466366e-02,9.86119211e-02,3.04830134e-01,3.23117256e-01,-3.77419263e-01,-6.94153532e-02,4.23110783e-01,-4.61187847e-02,4.99349952e-01,1.69336617e-01,1.82106256e-01,9.93302837e-02,-2.34050781e-01,3.55791748e-01,2.65761584e-01,9.28767845e-02,3.99907351e-01,8.01015943e-02,1.34822518e-01,3.06227177e-01,-7.48454213e-01,-4.11148816e-01,-3.91187251e-01,4.45996135e-01,-6.97568357e-02,2.06785113e-01,-4.89037126e-01,4.27726507e-02,-2.22780347e-01,5.65903246e-01,-2.00080916e-01,-3.18895578e-01,-6.16480231e-01,-2.75538832e-01,9.65693176e-01,3.57502848e-01,-3.40666100e-02,-2.02451557e-01,5.63324280e-02,-8.85901526e-02,6.32187277e-02,-2.67095715e-01,-2.72458076e-01,9.42475498e-02,-5.96332252e-01,-4.30868387e-01,5.13624847e-01,-2.58294903e-02,-5.18782020e-01,1.53936952e-01,1.50729254e-01,6.84211195e-01,5.88815331e-01,-6.17436409e-01,3.54861319e-01,-4.36583042e-01,-4.12458748e-01,2.90852815e-01,8.43976319e-01,-1.74154565e-01,-2.91549444e-01,8.65903914e-01,5.32554686e-01,-5.07389233e-02,-3.12784493e-01,-4.36928302e-01,-9.78833809e-02,2.71458626e-01,6.57177091e-01,-2.33842418e-01,-8.71215165e-01,-4.74676192e-02,3.50894690e-01,2.43539184e-01,7.10852742e-02,-1.56192612e-02,3.53489697e-01,-3.29113930e-01,4.94776815e-01,-6.33911252e-01,-1.34919569e-01,-2.50080317e-01,-3.80677879e-01,-3.73498470e-01,-4.19122845e-01,7.82921165e-02,-8.17287862e-01,-4.08441812e-01,-5.64266741e-01,-2.69622236e-01,-4.14675713e-01,-2.32593760e-01,3.97544652e-01,8.93596530e-01,-1.61212757e-01,1.25422165e-01,-1.73070077e-02,-6.79354846e-01,6.75417960e-01,1.96497902e-01,-1.88180208e-02,-2.22233623e-01,2.56757557e-01,-3.15306395e-01,6.32007062e-01,-4.34053332e-01,-2.16609687e-01,-3.16864371e-01,-7.11419702e-01,-3.19414765e-01,-6.04169786e-01,5.53009748e-01,2.70497345e-04,-3.24745268e-01,1.15053967e-01,2.25728586e-01,-6.28631830e-01,8.51998478e-02,-5.75308561e-01,-9.56453085e-02,2.73984820e-01,2.90817954e-02,7.48866424e-02,-2.11977199e-01,-2.46158451e-01,6.38535798e-01,-1.94838315e-01,3.82362336e-01,4.41476166e-01,5.03420532e-01,6.85855523e-02,6.24033332e-01,4.01197374e-01,-4.26869452e-01,-4.36496675e-01,3.83734077e-01,-1.38017340e-02,4.71825331e-01,2.68780589e-01,2.96145409e-01,-7.64629766e-02,7.81391039e-02,-8.00145790e-03,-1.56903282e-01,1.75876126e-01],[-0.18343097,0.1743192,0.07461832,-0.19982107,-0.04006213,0.5497848,0.06416616,-0.21108262,-0.6356132,-0.28916532,0.193581,0.0177111,-0.07040448,-0.5814121,-0.58210754,-0.06009458,-0.15677553,0.3160346,0.29838282,-0.45370692,-0.3094901,0.26274902,0.05644815,0.1797159,0.41414344,-0.31269,-0.07458381,0.345887,0.24907315,0.11573685,-0.2501085,0.518938,-0.23105122,-0.01388147,0.1486688,-0.16096367,-0.17891759,1.0518178,-0.10154412,-0.31245142,0.34476525,1.133283,0.6569955,0.18208298,-0.6880387,0.00662236,-0.06941147,-0.76681423,0.17396228,0.85788405,0.01603007,0.38788214,0.279318,0.50359035,-0.31215397,-0.17758678,-0.37796205,0.12139244,0.17734626,0.33997682,-0.2674575,-0.04507901,0.1314718,-0.18467623,-0.27602145,-0.2559302,0.16594557,-0.28791228,0.35676402,-0.47625953,-0.10604217,-0.7015373,-0.45834452,-0.22663929,-0.3025958,-0.48077872,-0.12279326,0.8067508,-0.10665288,-0.12942524,-0.07593027,0.09977669,0.42276633,0.02747046,0.3027619,0.13150494,0.00898813,-0.39753625,-0.04385259,0.9909243,0.20236291,-0.15374166,0.50215626,-0.39336613,-0.03768967,-0.0342825,0.03046781,0.05711506,0.16803639,-0.18579629,0.5139328,0.23873082,0.26496768,-0.10774599,0.4130523,0.04351586,-0.6375272,-0.15783307,0.00895195,0.0775414,-0.10262583,0.33211768,0.31100357,0.4575319,0.05827974,-0.14640279,-0.6662887,-0.07575404,0.16298915,0.6551511,0.32989737,-0.51075035,-0.13558112,0.3211188,-0.37008825,-0.25889713,0.2280812,-0.25577033,-0.02970781,0.130103,-0.06404972,0.01471611,0.40697917,-0.09906885,-0.39801744,0.21027558,-0.120562,-0.31908062,-0.22306377,-0.08199457,0.10837121,-0.03087071,-0.26158708,0.15411617,0.14247577,-0.04456145,-0.35854042,0.11437494,0.12067667,-0.20500243,-0.39646286,-0.38790485,0.8774395,0.8359688,0.05991519,0.18049265,0.49048743,0.78022146,-0.10805614,0.84569794,0.09506348,-0.15966707,0.03468018,-0.01350303,-0.66334295,0.02996831,-0.21400014,0.71184,-0.17364012,0.6341653,0.18573985,-0.15012927,0.51125747,0.4125514,0.17938809,0.06538959,-0.32550246,0.03606968,-0.30499965,0.04810934,-0.07696155,0.3039832,-0.14621978,0.08412672,-0.5646161,0.82228076,0.69878787,0.10578094,-0.3700774,0.66905683,-0.16062379,0.25463507,0.3946845,0.09087569,-0.4584444,-0.35699815,-0.42860952,-0.01644943,1.3915781,0.21244183,-0.97049254,0.36312842,-0.5269479,-0.39848056,-0.2249671,0.35558423,0.15285648,0.32689723,0.15802553,0.48512548,-0.57016706,0.2256573,-0.35487944,0.03829048,-0.12450793,-0.17887263,-0.1955734,0.31288695,0.20805393,-0.64918494,-0.07008865,0.5161279,0.4761959,-0.5347079,-0.09782038,-0.05774533,-0.08087067,-0.20811138,0.45943007,0.39388898,-0.22043915,0.15826033,0.37678972,-0.0429537,-0.08455844,0.15289585,0.29414302,0.04379233,-0.07603759,0.25107503],[-0.22583371,-0.92831576,0.0403181,0.13258006,-0.04891467,-0.15748541,-0.02427776,0.34753942,0.26131904,-0.15860957,-0.36529404,-0.24933207,0.04391815,-0.18572196,-0.1372513,0.21713221,-0.3120023,-0.29645652,-0.41113564,-0.04000121,-0.06236742,-0.9597719,0.5872088,-0.18886778,-0.48885372,0.65963686,0.4287583,0.22644007,-0.07560338,0.5640969,0.34036013,0.2797875,0.02206075,0.19419709,0.746415,-0.8016303,-0.66373676,0.3789754,-0.20086648,-0.3548541,-0.28074285,-0.21808903,-0.48142138,0.28765726,-0.18219346,-0.26684266,-0.3386195,-0.5197925,-0.23607329,-0.48711607,-0.4956042,-0.367438,-0.09941623,0.4546155,-0.09507474,0.20115569,-0.2609643,0.37936258,-0.37882176,-1.085219,-0.6154617,-0.66182876,-0.07063224,0.03935806,0.5943647,-0.11159918,0.3912061,-0.2640553,0.26516837,-0.33500466,0.18949856,-0.06594224,-0.10512052,0.34925476,0.4233727,0.08100604,0.5871414,-0.03914276,0.6789717,-0.8157768,-0.3000853,0.06585155,0.43485114,-0.4635752,-0.00403272,-0.3551186,-0.42447165,0.8098062,0.27896586,0.1405054,0.07554707,0.26336583,0.4953189,-0.3909972,-1.0338776,-0.2098501,-0.06373354,-0.27577257,0.2376927,-0.617241,-0.8772662,-0.2587799,0.5964379,0.06483807,0.3443386,-0.0138791,-0.5430396,-0.8034082,0.2837522,-0.28476024,-0.08253843,0.04484092,-0.54097146,-0.08875781,0.2157486,-0.73514456,-0.3244046,0.57849413,-0.00272746,1.0056221,0.49148244,1.2966205,-0.44936937,0.18710102,-0.13482836,-0.31014645,-0.44718993,0.822507,0.90505004,-0.54376554,0.4374067,0.43362007,0.2754676,0.03961271,0.23092175,0.32796744,-0.02314315,-0.23745212,0.11517924,-0.5279961,-0.16110706,0.5855777,0.05997068,0.40214053,0.0381956,-0.23547617,0.02408737,-0.02309312,0.37995365,-0.0628458,-0.5742294,-0.35535312,-0.4107346,0.34733787,0.00392651,0.20268112,-0.28599852,0.43875274,0.57932353,-0.78504026,0.02612062,-0.20937057,0.00801933,0.5647487,-0.945657,-0.26417023,0.06304334,0.42882094,0.13118176,-0.20910092,0.00680925,0.62721384,0.04168948,-0.00499314,0.44772685,-0.67203337,-0.63786095,-0.72004175,-0.23408328,-0.08147287,-1.0101081,0.13027754,0.28229728,0.09369463,0.17517295,-0.02426689,0.2002327,0.3300926,0.22694016,-0.21585809,0.18600091,-0.32329887,0.02332481,-0.07360249,0.10353382,0.06167926,-0.12126099,0.41740385,0.1079103,-0.15926686,-0.61142933,0.7360118,-0.316574,0.14797367,0.10300681,-0.78281915,-0.33833385,0.48157606,-0.42974782,0.55577487,-0.26083088,-0.44165564,-0.4575539,0.21961354,0.27965832,-0.02131849,0.40195423,1.0224326,-0.12889983,0.48902208,-0.2040875,-0.49093315,0.09544062,0.04580472,-0.7082679,-0.2381075,0.30743498,0.84826934,-0.25047833,-0.5501564,0.04094892,0.1704783,-1.2550036,0.01680321,0.33940366,0.17463306,0.01901738,0.1813825,-0.17818998,-0.02551511],[4.75786358e-01,4.09446567e-01,3.51485252e-01,2.40130397e-03,-7.23160505e-01,-3.10374349e-02,-3.68734300e-01,-1.75850004e-01,-1.18033695e+00,6.98747993e-01,4.93072540e-01,-5.56172848e-01,-3.57031643e-01,2.04773024e-01,-3.37550193e-01,2.83138622e-02,5.03492681e-03,4.30590510e-01,2.36168385e-01,-8.88246670e-02,-3.58319640e-01,1.10136569e+00,-9.52226147e-02,3.41756791e-01,-4.51065063e-01,3.11079919e-01,-9.48852539e-01,-4.89656210e-01,-6.29094392e-02,2.68794090e-01,-1.09755135e+00,4.86726612e-01,-3.45906973e-01,-4.83212918e-01,5.49262524e-01,-1.65039733e-01,1.25017539e-01,-6.58754230e-01,-1.03106521e-01,1.84398249e-01,6.17106140e-01,1.52322367e-01,-1.32906288e-01,-8.36295128e-01,-2.97877610e-01,1.88666567e-01,-3.19199890e-01,-6.61712110e-01,-1.45729139e-01,9.97327119e-02,-2.05372229e-01,5.81198215e-01,4.17624652e-01,-8.94274712e-01,6.31920770e-02,1.36214435e-01,-1.99191701e-02,5.67997843e-02,1.53292075e-01,2.80747801e-01,1.13631792e-01,-7.53814122e-03,-4.19669658e-01,6.04853928e-01,-9.34877619e-02,-2.83013564e-02,3.55783373e-01,3.97937357e-01,-1.87223583e-01,-5.03595710e-01,-3.29120219e-01,-2.12820042e-02,-1.43195689e+00,9.73065495e-02,3.74024391e-01,-9.49602425e-02,3.46602708e-01,-1.00802672e+00,-5.34588933e-01,-1.00990558e+00,4.21146214e-01,7.03092337e-01,-1.30928874e-01,-7.50509650e-02,1.13661714e-01,-4.67930257e-01,-4.72645760e-01,1.44491225e-01,-4.69513200e-02,3.00607644e-02,1.75767019e-01,2.52077967e-01,9.43131447e-02,-8.06528032e-02,1.03763424e-01,1.17172182e-01,-3.18874449e-01,-3.46091986e-01,-2.30676547e-01,-2.90147334e-01,-7.45948315e-01,3.34937334e-01,4.94019464e-02,-3.04562617e-02,-5.64065754e-01,1.66700840e-01,2.66378243e-02,2.72870541e-01,5.49164951e-01,-4.00870562e-01,7.66386271e-01,-1.17639981e-01,-6.13297880e-01,-1.66712746e-01,-4.63704586e-01,5.13937593e-01,-2.53600869e-02,2.44202182e-01,7.22980797e-01,-7.28644013e-01,-3.03783238e-01,2.53371209e-01,-2.51248628e-01,-2.51906633e-01,3.30784559e-01,-4.84817475e-01,7.11854339e-01,2.08351180e-01,-5.50942540e-01,1.93627834e-01,-3.58528495e-01,-2.23423779e-01,-4.55145314e-02,-6.72961473e-01,-3.31286430e-01,2.36684475e-02,-1.09111100e-01,-2.77194250e-02,-1.83305681e-01,1.18654035e-01,3.38208936e-02,-6.52738094e-01,-1.02832043e+00,-1.13543129e+00,3.14959019e-01,-6.62873447e-01,6.35819808e-02,1.40110537e-01,3.74630928e-01,6.99254692e-01,2.01970153e-02,-5.50329208e-01,-8.94988254e-02,5.12290776e-01,-1.86616741e-03,-1.40277147e-01,5.01337051e-01,-2.82200575e-01,-4.92444962e-01,-2.83200443e-01,-3.97262722e-01,1.63703084e-01,-3.85445237e-01,6.32915437e-01,2.47486025e-01,-4.69255090e-01,2.38915533e-01,1.02907503e+00,-4.40230489e-01,7.30862021e-01,-3.35178196e-01,2.60763429e-02,3.71544659e-01,-3.68700027e-01,-2.18804345e-01,1.57083601e-01,5.70350587e-01,1.82425365e-01,6.64936262e-04,-8.15967917e-01,5.34334853e-02,3.34348708e-01,-7.47636497e-01,4.92432117e-01,-1.87019855e-01,-1.84390083e-01,5.29933035e-01,-3.42916369e-01,-2.96043873e-01,-4.23968524e-01,5.92338443e-01,-9.60362911e-01,4.62732129e-02,1.92160636e-01,8.36133212e-02,6.71402395e-01,-3.84867638e-02,-2.75156826e-01,7.69117055e-03,-1.32237196e-01,-7.83609927e-01,-1.05115965e-01,-1.98636666e-01,-3.22033793e-01,1.56477187e-02,4.93349463e-01,-2.63142828e-02,-1.75970837e-01,2.82037050e-01,-2.93272108e-01,1.09732606e-01,-9.01941583e-03,-3.04481626e-01,1.47697330e-01,2.06061259e-01,4.43582892e-01,1.34693533e-01,4.96007651e-02,-7.03404665e-01,2.41910785e-01,-1.69295166e-02,-9.37974602e-02,8.03963542e-01,-2.26595789e-01,2.08466947e-01,-7.44343698e-02,-3.75627488e-01,6.58804417e-01,2.13733479e-01,-1.00119042e+00,1.97317481e-01,8.45639706e-01,-5.77982627e-02,4.11080658e-01,1.00193715e+00,-6.23233855e-01,-5.46081960e-01,1.21007413e-01,-5.31185605e-03,2.33106196e-01],[-1.64529487e-01,1.66750401e-01,2.75529504e-01,1.49908438e-01,-3.97380218e-02,-4.10375781e-02,2.07507506e-01,1.01736486e-01,-5.87798417e-01,3.64331216e-01,-5.67629695e-01,3.65065098e-01,2.92775780e-01,3.37551564e-01,-6.34281456e-01,2.88004905e-01,3.96471560e-01,1.25812769e-01,-4.95665640e-01,3.15759867e-01,4.78534773e-02,-4.91738200e-01,-5.57919919e-01,-8.54549348e-01,1.49275810e-01,-7.07002580e-02,4.23290342e-01,3.78860712e-01,1.46447942e-01,-4.94173378e-01,5.84034659e-02,-5.11679530e-01,4.13953900e-01,-6.08983159e-01,-2.59755645e-02,-1.30217046e-01,2.05713436e-01,5.79988778e-01,1.53301939e-01,5.60432300e-02,-8.79693627e-02,-4.90741730e-02,7.23405838e-01,3.81964654e-01,-3.81928504e-01,4.76065159e-01,4.79731977e-01,-4.86492038e-01,-8.03741813e-01,9.69067693e-01,-1.63191631e-02,2.41425008e-01,6.97505996e-02,-1.50576365e+00,-4.87875730e-01,-1.79547548e-01,2.93264896e-01,4.67287242e-01,-2.32886940e-01,3.03346723e-01,-7.41926432e-01,-2.58231372e-01,-2.35769942e-01,1.46771163e-01,7.24723488e-02,1.83503151e-01,4.15008157e-01,-6.91869915e-01,2.14990333e-01,6.21404648e-01,1.29861291e-02,3.23466539e-01,-7.13587582e-01,-3.51268619e-01,-1.14336438e-01,-2.26179615e-01,-9.72694039e-01,4.56119537e-01,1.39452234e-01,-3.82066756e-01,-2.62999386e-02,2.60698974e-01,-9.38006490e-02,-6.55258417e-01,-4.29529697e-01,-1.46055624e-01,-2.27189481e-01,-6.78256452e-02,-1.34350821e-01,6.01363182e-01,-1.11467414e-01,9.97251272e-03,2.82706857e-01,-4.27219808e-01,-4.94421095e-01,4.76640582e-01,-6.18305743e-01,9.51002479e-01,3.35786492e-01,1.03144437e-01,-3.59641582e-01,-1.12670071e-01,-8.33342671e-01,1.32556722e-01,5.45621037e-01,7.22386539e-01,9.24466411e-04,3.03050727e-01,7.56725430e-01,-5.47673583e-01,-3.52137148e-01,-6.17161930e-01,3.35378408e-01,9.78737101e-02,-2.87327226e-02,-5.95216274e-01,8.26104954e-02,-2.11527452e-01,1.38959020e-01,-1.18455756e+00,-3.69554162e-02,-1.27983063e-01,3.96903992e-01,7.83745199e-03,-9.87341069e-03,-3.26305404e-02,-6.21551182e-04,-6.59550428e-01,3.73358801e-02,2.25548103e-01,1.43122338e-02,2.62571275e-01,2.18615249e-01,3.22688669e-01,1.99122019e-02,-3.63554537e-01,1.08142056e-01,4.41613048e-01,1.45430148e-01,-4.85122383e-01,4.42374557e-01,-5.26445448e-01,-2.43156984e-01,2.24018946e-01,2.79596359e-01,-2.04683393e-01,-1.70636028e-01,-9.41038802e-02,4.96478617e-01,-3.43073457e-01,-2.11313978e-01,-2.71754265e-01,-3.12933978e-03,-1.01768393e-02,-2.56660193e-01,-1.10601462e-01,2.32986480e-01,-5.66547178e-03,6.89824969e-02,-3.59747231e-01,5.49083471e-01,1.24404855e-01,2.63726741e-01,-4.93132383e-01,-2.09895641e-01,2.77722776e-01,-2.42849529e-01,-5.44327557e-01,-3.49264413e-01,-5.05189538e-01,-4.86513853e-01,3.33432527e-03,-6.10035472e-02,-6.85430542e-02,-4.26128715e-01,-5.20183980e-01,-2.82120436e-01,1.43691093e-01,-2.75733322e-01,-4.96564180e-01,2.57165432e-01,-7.18118548e-02,-3.88119340e-01,-2.91910619e-01,2.26257831e-01,-5.15039682e-01,4.87343282e-01,6.57122016e-01,-1.17330492e+00,8.80988002e-01,1.01552662e-02,-8.85806698e-03,5.27508855e-01,1.74490452e-01,-5.85094333e-01,-5.44120595e-02,-5.72452009e-01,4.86514956e-01,1.20903686e-01,-1.84338689e-01,-8.49040329e-01,-1.22085407e-01,-2.66765773e-01,-5.21466844e-02,4.63677406e-01,-1.37309462e-01,2.28180423e-01,3.50958794e-01,1.44757405e-01,1.68852597e-01,-3.10214639e-01,-4.43242431e-01,5.53517938e-01,-2.55934317e-02,-8.81208777e-01,-4.03446943e-01,-2.69410580e-01,2.91952521e-01,3.53249490e-01,4.36248928e-02,3.95570666e-01,-1.85214654e-01,-8.50879252e-01,5.05523801e-01,2.84335762e-01,4.97818105e-02,2.89418161e-01,6.66223884e-01,1.00520357e-01,3.28408182e-01,1.07689053e-01,-3.92261416e-01,2.31193617e-01,-1.80230469e-01,4.91031796e-01,4.48627084e-01,2.40753040e-01,1.25158608e-01,2.81531304e-01,8.16434920e-02],[4.94341373e-01,-1.95911318e-01,-9.73053098e-01,2.53049403e-01,-1.47224203e-01,-9.13918316e-01,5.11119843e-01,-1.55435875e-01,-5.86301982e-01,2.12939397e-01,5.14457524e-01,-1.69480532e-01,-3.63538057e-01,4.14087653e-01,3.29514325e-01,7.91247338e-02,2.57972062e-01,-1.51631296e-01,-3.42356712e-01,-1.25426233e-01,4.60938126e-01,2.29196623e-01,-2.24849135e-01,-4.34463859e-01,-1.54672161e-01,5.44414878e-01,4.31151092e-01,-2.20493019e-01,3.76040518e-01,5.24544775e-01,3.09008136e-02,-2.40716383e-01,-2.78443508e-02,-3.16894740e-01,-1.17038250e-01,3.84942442e-01,3.15450311e-01,-3.50007415e-01,-4.55567867e-01,2.36646906e-01,-3.58745784e-01,2.73305833e-01,4.52225268e-01,1.23210356e-01,3.80650610e-01,-1.26788333e-01,-2.59095430e-02,-2.91182734e-02,1.28735960e-01,5.11410952e-01,1.83555722e-01,-4.47866529e-01,3.21306497e-01,4.25729789e-02,6.18685663e-01,-6.98042847e-03,2.14007840e-01,2.04367250e-01,5.06710529e-01,5.89411914e-01,-1.40534282e-01,4.09787297e-01,1.52849451e-01,-7.92696238e-01,-5.54603040e-01,-2.12160453e-01,-3.17471474e-01,-5.55102527e-02,-3.63350093e-01,1.43755823e-01,-4.14667457e-01,-2.20086321e-01,-3.07611346e-01,-5.87146103e-01,-1.80824444e-01,6.23447478e-01,-1.08573830e+00,8.24122250e-01,1.21056728e-01,-5.71389556e-01,1.12155378e+00,-4.22967255e-01,1.52291283e-01,-5.50053179e-01,-1.41936643e-02,1.41122863e-01,4.03915018e-01,2.17637390e-01,1.12932861e-01,-1.25869378e-01,-1.98764414e-01,-1.01203322e-01,-1.98714770e-02,1.27627272e-02,-2.07201079e-01,3.30816090e-01,1.08151667e-01,-2.64737755e-01,-1.05504971e-03,5.59836514e-02,2.36880377e-01,-7.77959049e-01,-1.40732259e-01,-7.40737393e-02,-3.41593772e-02,-7.74734735e-01,-3.48603338e-01,-8.28611664e-03,1.82875603e-01,-3.88407439e-01,6.63283765e-01,-6.74512088e-01,-4.03180927e-01,1.25327989e-01,-3.33852619e-01,-6.14311755e-01,6.99158609e-02,-6.28993452e-01,9.22971487e-01,-2.95759857e-01,-2.41184577e-01,2.64174014e-01,4.07356024e-01,-2.48949677e-02,-5.10892458e-02,-3.03674042e-01,1.85078800e-01,2.27154657e-01,3.81369859e-01,-9.68602657e-01,-3.43268141e-02,-1.35968432e-01,4.47859198e-01,-1.02781022e+00,1.33046001e-01,-9.31903347e-02,-5.56869805e-01,-6.14919722e-01,9.87554044e-02,5.58925718e-02,-1.32365048e+00,2.34812722e-01,-5.85553646e-01,-2.54683524e-01,7.42357969e-01,-3.03558959e-03,1.00784227e-01,4.45252270e-01,-1.03800721e-01,-3.30960512e-01,-2.05968525e-02,2.38686815e-01,6.80662841e-02,-7.97365531e-02,6.14327550e-01,1.53123349e-01,8.64241049e-02,3.14379215e-01,3.04871976e-01,1.61795571e-01,-2.35272780e-01,-2.59051114e-01,-1.77411432e-03,4.22218204e-01,-4.48820621e-01,-9.48587656e-02,7.80220702e-02,3.00897628e-01,2.23500580e-01,-9.33695436e-02,2.21682675e-02,3.40658128e-01,5.85293397e-02,-3.89890783e-02,1.02321945e-01,-1.23617440e-01,-5.98544657e-01,-1.71298251e-01,3.54733437e-01,3.63553584e-01,5.88126063e-01,4.48997110e-01,-1.06124841e-01,-6.75044537e-01,-1.96891651e-01,2.09670529e-01,4.91149008e-01,4.77697194e-01,-2.67772973e-01,3.59962553e-01,9.88704860e-02,1.62468866e-01,7.83564672e-02,-3.55891705e-01,3.27863097e-01,2.14801639e-01,7.88525939e-01,4.27159742e-02,5.60424030e-01,-3.76321329e-03,3.03055316e-01,-3.16296518e-02,-4.38747406e-02,7.24842921e-02,-3.44721228e-01,1.23128809e-01,-5.06387889e-01,5.99514365e-01,-2.67364606e-02,3.41618396e-02,-6.45597398e-01,-5.39706834e-02,-5.17450452e-01,3.65026087e-01,-4.85797197e-01,1.01720113e-02,4.07559782e-01,-9.54270810e-02,-8.69851768e-01,1.08782016e-02,-1.04093291e-01,5.51159561e-01,-5.20298667e-02,-1.26477015e+00,4.17132586e-01,-1.47937210e-02,6.31299973e-01,-8.74587595e-02,5.35387933e-01,-3.46960217e-01,2.11460620e-01,-1.60629153e-01,4.02342886e-01,1.37150781e-02,-2.12643072e-01,1.00848623e-01,-3.58377725e-01,-4.77258593e-01,2.44559973e-01,-1.08445168e-01],[-0.48413104,-0.42980823,0.04274462,0.64168334,-0.11093175,0.10139795,0.5611933,-0.29943156,-1.1388085,-0.59767544,0.39027083,0.39175165,0.34236196,0.02248852,0.5898908,0.0218445,0.38818768,0.08478712,-0.41301137,-0.9531022,-0.31931195,0.7958769,0.06249178,-0.17800185,0.96197444,0.38688028,0.01404904,0.10116704,-0.22114225,0.33479142,-0.22224395,0.620647,0.2408877,0.14264613,-0.322224,-0.06972828,-0.25573483,0.24078314,0.48780665,0.13681959,-0.3766097,0.19626668,-0.1218358,0.56667864,0.16675565,-0.25049374,0.09174987,0.17885402,-1.2679194,-0.25031343,-0.06733371,-0.46831843,-0.4770795,0.42053935,-0.2836466,0.12122749,-0.6466323,0.5147292,-0.3326544,0.54373825,-0.46919817,-0.18903568,-0.32790843,0.27556998,-0.721037,0.14424764,0.21434957,-0.51085323,-0.03475459,0.3879461,-0.4096735,0.38452002,0.43566966,-0.40992022,0.81467074,0.22004518,0.40311208,0.19974892,-0.26180023,0.27892777,0.06681761,0.20891707,0.9310384,-0.4537384,0.6141337,0.10149692,0.03288975,-0.33007035,0.6670304,0.3267865,0.19236423,-0.5889542,-0.5728088,-0.2689436,0.1570804,0.2553535,-0.45197234,0.2605881,-0.0840036,0.04897787,-0.33571967,-0.30674106,-0.11210217,-0.09262784,-0.05923177,0.18568951,-0.40062526,-0.69531703,0.5301934,0.2825719,0.6373333,0.22023016,0.6039048,0.05431084,-1.0420372,-0.11557476,-1.390913,-0.06834044,0.33501387,0.15579776,-0.33940908,-0.2772269,-0.05222411,0.07949603,-0.4439908,0.2560898,-0.4136154,-0.32091662,0.02439042,0.02566001,-0.08364722,-0.20841971,0.30581588,-0.07435826,-0.81642306,-0.1096816,-0.59096676,-0.12277769,0.49482915,-0.808024,-0.35232884,0.19461763,-0.7601361,0.0343841,0.10848281,0.29733407,0.11053329,0.01750914,-0.4434321,-0.22465718,0.49731544,-0.05095835,0.08193801,-0.19984628,-0.10046417,-0.11794724,-0.05966251,0.2180512,0.3991453,0.09951437,-0.0486854,0.41836253,-0.06525311,-0.27629092,0.05991685,-0.19665794,0.31830037,0.7715219,-0.14978923,-1.0472642,0.26143995,-0.4553121,0.02518753,0.24410522,-0.36799195,0.24397406,-0.0634801,0.30690837,0.19466555,-0.2463429,-0.49832413,-0.48726404,-0.12151739,-0.32941273,-0.3677949,0.41739017,-0.64759195,-0.3329953,-0.45271406,-0.03865491,-0.4382006,0.21541063,0.13642628,0.6845779,0.42079931,0.4300457,0.33817825,0.11748096,-0.0416468,-0.24887815,0.0824753,0.5990018,-0.3212795,0.05181087,0.10073194,0.07666302,-0.35852703,0.27305403,0.34046134,0.06086671,0.8567705,-0.6889642,0.46575066,-0.24681644,0.12110233,-0.19430164,0.23202713,-0.6706694,0.41768724,0.15462658,0.7997916,0.10318447,0.0776083,-0.04018855,-0.9035882,-0.4818995,-0.17376137,-0.49954605,0.10838964,0.5306528,0.17620593,-0.06806034,0.03400399,-0.6502547,-0.4910058,-0.5033477,0.7291295,-0.11735138,0.58817387,0.6514727],[-0.64711285,-0.10880372,0.4138241,-0.77539074,-0.33052507,-0.6916097,0.04800849,0.6008805,0.01999601,-0.3826769,-0.15482555,-0.0548376,0.4669223,-0.27170354,0.49706802,0.12042385,0.26885328,-0.03967294,-0.1675145,-0.21604328,0.31122488,-1.0575316,-0.20543076,-0.5309334,0.21928097,-0.12896971,0.18728966,0.51173747,-0.7814482,-0.32219818,0.3792018,-0.17614768,-0.13413447,0.757585,0.33361155,-0.21286115,0.58608013,0.18474373,0.04956956,0.38785776,0.16325015,0.3431379,-0.9314423,0.4237783,0.51257026,0.18751441,-0.42388275,0.45548683,-0.5286163,0.08776519,0.07476763,-0.11219099,-0.30736652,0.3231668,0.3069463,0.43523842,-0.1954949,-0.13876754,0.65804905,-0.5976475,0.83317345,-0.04200114,0.38220835,0.20932871,-0.14324187,-0.22480851,-0.7278149,0.36234426,0.4474355,0.3599493,0.21332039,-0.08725207,-0.5881857,0.11172683,-0.05007359,-0.19901004,-0.4654253,-0.06548642,0.01844375,-0.24565487,0.3891266,-0.58965683,-0.18195355,-0.32136595,-0.18297642,0.12443658,0.08494349,-0.46308514,-0.598948,-0.28353402,-0.5353898,0.9387662,-0.24888459,-0.12320512,-0.6656186,0.57587457,-0.25331587,-0.22871017,0.10433336,-0.30582148,0.0169786,-1.2001064,0.29275072,-0.35086325,0.35671902,-0.19595636,-0.22860269,0.75357646,0.49261495,-0.47262022,-0.41565382,0.04621486,0.4319086,0.87166387,0.06903379,-0.667807,0.2737069,0.2881479,-0.08506816,0.49552667,-0.5188424,-0.40181687,-0.43330225,0.13113977,-0.7682925,0.13196273,-0.17668734,0.19222566,0.2903037,0.16957611,0.07777765,0.4749483,0.4982814,0.01565457,-0.731518,-0.38883263,0.17402479,-0.21454689,-0.09553078,0.42580882,0.5768753,-0.21993963,0.2366729,0.03744837,0.11668151,0.19281223,-0.36151263,-0.19881919,-0.18976611,0.05027307,-0.20910221,-0.02363833,-0.02726894,-0.04445583,-0.28170195,0.04703509,-0.19200553,0.06752324,0.10472786,0.27395028,0.6315801,-0.9470647,-0.00606506,-0.1848732,-0.75501996,-0.74166465,0.29557428,-0.39519238,-0.3298385,-0.6256262,0.01025935,-0.19245692,-0.02882085,0.09517366,0.2873865,0.01183752,-0.09542978,0.21734124,-0.15855955,0.2817852,0.28362036,-0.7392786,-0.06880458,0.38141116,0.89829826,-0.12744597,0.01625278,0.6428917,0.17770453,-0.21586618,0.30007744,-0.4240835,0.14511617,-0.5072171,-0.63368165,0.33935815,0.22770843,0.4032641,-0.15512578,0.6127773,0.18576638,-0.25225708,0.12787975,-0.4748337,-0.06937426,-0.26038802,-0.61890525,0.90538055,-0.51925373,0.53676873,0.0932459,-0.29103854,0.39415073,0.20294191,0.1249277,0.45081833,0.2954475,-0.8237842,0.01301672,0.02314059,0.28830934,0.2557959,-0.09003328,0.9835793,0.24219264,0.1499737,0.45535505,0.2592511,-0.14962818,-0.00730865,-0.09911767,-0.21398185,-0.37768823,0.33427194,-1.1085801,-0.26876915,-0.14817405,0.01026567,0.7129649,-0.4612861],[-3.94564569e-02,7.64388502e-01,-1.66353281e-03,-6.64785653e-02,-4.12453502e-01,1.35407373e-01,1.21882297e-01,-4.41553473e-01,2.13764347e-02,3.08019757e-01,8.18271339e-02,-7.51034260e-01,-7.49541745e-02,8.89758408e-01,6.88489974e-01,4.27524954e-01,-5.59061825e-01,9.03247476e-01,-1.81761205e-01,7.37028643e-02,-5.79911411e-01,6.51532233e-01,-8.26312602e-01,2.26525575e-01,-7.54674137e-01,6.16669133e-02,-6.80922985e-01,-4.62521881e-01,-3.73408943e-01,1.14831641e-01,-3.90372008e-01,-2.38919377e-01,-9.84037369e-02,1.29180610e-01,2.15174884e-01,2.12391034e-01,-2.33921006e-01,1.21851578e-01,3.07794869e-01,-6.56341195e-01,2.40511090e-01,-1.69529811e-01,-7.36494586e-02,-5.92161179e-01,-1.86002657e-01,-7.51147747e-01,-7.61805415e-01,8.85819674e-01,-2.98032109e-02,3.51253420e-01,-4.13333625e-01,5.29665828e-01,2.16062054e-01,-4.04763669e-01,3.94392610e-01,5.91015816e-01,-4.17348891e-01,-2.45079890e-01,8.42156827e-01,4.39842373e-01,6.42323077e-01,-3.66212539e-02,3.84711176e-01,2.56961346e-01,-6.82578027e-01,-2.34634787e-01,-6.08217716e-02,1.09619178e-01,4.02861267e-01,-1.54075563e-01,-8.52429420e-02,1.01618487e-02,-2.10154831e-01,-7.28273273e-01,7.56927967e-01,7.95144498e-01,8.63955542e-02,-6.34302497e-01,8.09585512e-01,-1.63640544e-01,4.44610804e-01,-5.20422935e-01,-4.04112726e-01,-7.70903938e-03,3.70686084e-01,-7.02333331e-01,-5.03515780e-01,6.64487243e-01,-1.00245285e+00,-7.84532428e-01,1.48026764e-01,3.60287756e-01,-6.28683791e-02,2.42394030e-01,1.20487489e-01,-2.51307338e-01,3.17169540e-02,7.58737206e-01,-5.97551703e-01,7.53646851e-01,-2.49964610e-01,-8.67429376e-01,-2.11302653e-01,7.08356202e-01,2.50883222e-01,-2.13674918e-01,1.01189129e-01,-1.13473237e+00,-3.28659564e-01,-7.19659507e-01,2.48837247e-01,-2.24471539e-01,-8.12551454e-02,1.28817275e-01,3.45883280e-01,-8.68491158e-02,-2.41482571e-01,-3.69291037e-01,5.52553058e-01,-5.65043092e-01,1.59553364e-01,-1.05228031e+00,-5.13017476e-01,-6.23372085e-02,-3.12829316e-02,4.43381779e-02,-3.12013388e-01,5.24664149e-02,-1.86917156e-01,-2.10687310e-01,-2.66920596e-01,1.76985621e-01,-4.52785164e-01,3.70719612e-01,3.53720456e-01,-7.17446566e-01,-1.16994321e+00,-1.25407100e-01,-4.64578897e-01,5.53960741e-01,1.97494659e-03,-1.01863563e+00,-1.12619781e+00,-6.79775849e-02,-1.05445506e-02,-7.26964548e-02,9.33395207e-01,-9.16283298e-03,5.02259910e-01,-2.35312715e-01,3.11003029e-01,-1.13835788e+00,-4.42329586e-01,-4.64625508e-01,-5.70615411e-01,-7.52041459e-01,7.06682622e-01,2.31603876e-01,1.23130612e-01,-4.55256030e-02,3.83856803e-01,-3.75125736e-01,-8.76633599e-02,-1.41714849e-02,-1.58767402e-01,-4.55025196e-01,-7.90559351e-02,-2.95298308e-01,9.87796560e-02,3.93831223e-01,-7.53842890e-01,-2.29617301e-02,-2.30248615e-01,-7.86375031e-02,-1.45910129e-01,-1.73581596e-02,-1.91563413e-01,2.72349805e-01,7.98072889e-02,4.90628123e-01,-5.72305083e-01,-3.24200213e-01,-2.54870832e-01,1.05036998e+00,2.41874643e-02,-6.78370297e-01,1.18107907e-01,4.73157078e-01,5.44278808e-02,-8.85706395e-02,-2.71663696e-01,6.16354585e-01,2.19961300e-01,-2.53752768e-01,2.86603600e-01,-2.99547046e-01,2.26089224e-01,-5.86231291e-01,-5.19481361e-01,-2.76770353e-01,-2.09186599e-02,-1.83115512e-01,-1.15533672e-01,-3.09937149e-01,-3.31392676e-01,-6.50663720e-03,-9.66041386e-02,5.73220134e-01,-1.79429919e-01,2.86513031e-01,3.10383350e-01,5.05962551e-01,2.10639909e-01,5.07803321e-01,-4.90761518e-01,6.80505574e-01,-4.36627477e-01,-5.33104181e-01,4.32010330e-02,-4.41033065e-01,3.33339609e-02,4.96606320e-01,-4.51299727e-01,-3.03000271e-01,1.33721143e-01,-3.78705859e-01,7.36796558e-02,2.57647131e-04,-1.18254796e-01,9.84105349e-01,-5.02128661e-01,-6.94486797e-01,1.14528567e-01,-5.95493555e-01,-4.27133948e-01,-7.66528249e-01,-6.58256233e-01,3.17589706e-03,-8.09930786e-02,3.04374456e-01],[0.20578077,0.24720214,-0.16849634,0.3013507,0.507479,-0.28640574,-0.7055494,0.46407986,-1.7999415,0.03363954,0.26878428,0.15191178,0.73351175,-0.21100229,-0.0225528,-0.35926706,0.25056323,-0.30814496,-0.84527254,-0.37008548,-0.6005831,-0.05318915,-0.9768053,0.36251932,0.23121023,-0.43318588,0.00780883,0.13378109,0.38232285,-0.17640457,0.1217888,0.6721169,0.10901766,0.07364,-0.02633881,-0.6964144,-0.58155584,0.32243577,-0.40195933,0.2060414,0.15937445,-0.24867962,-0.1324957,0.00922353,0.44919384,-0.1758965,0.42471057,0.5712412,-0.10305432,-0.00629137,-0.46397415,0.55729884,0.18178932,-0.3717493,-0.22268471,-0.81280404,0.47992957,-0.09358914,-0.6830775,0.33765265,0.5804465,-0.57609236,-0.29732984,-0.05301649,-0.03206181,-0.12440891,0.23345472,-0.50163215,0.13340077,0.1021691,-0.41601804,-0.51451916,-0.40802926,-0.18938178,0.06485211,0.6209407,0.3140807,-0.3485168,-0.8101845,-0.8377783,-0.06514673,-0.72187376,0.357903,-0.01755668,0.5361789,-0.42909995,0.13705045,-0.30969393,-0.40828452,0.14490336,0.51817596,-0.01165146,-0.04033694,-0.09488266,0.2729978,0.41209272,-0.06815735,-0.4706046,0.2563084,-0.25836325,0.182149,0.22937405,-0.47658193,-0.34873044,0.74608475,0.25715223,-0.7290449,0.10194293,0.23498288,0.10003169,1.0534924,0.5133218,-0.6311445,0.15751028,-0.64586675,-0.43298995,0.09346021,0.5817844,0.2729896,0.10806847,-0.06402081,-0.007839,0.17191148,-0.21938683,-0.10947824,0.6825447,0.9707796,-0.6561314,-0.5035959,-0.35305452,-0.47068027,0.48884714,0.34558392,-0.4400878,0.36519006,0.12647955,0.5694924,-0.21052094,0.50249815,-0.09791039,0.02714779,-0.5261869,-0.06215312,0.40015867,0.48966125,0.28349775,0.1576596,-0.13625965,0.00972733,-0.03005111,-0.36864388,-0.43375203,0.13016495,0.25979525,-0.10916311,0.33517826,0.37625194,0.5665746,0.42207256,-0.40744358,-0.3127181,-0.10421109,-0.06990642,1.0571755,-0.30176294,0.08675536,0.0877939,0.19009486,-0.02680973,-0.2386526,0.01796661,-0.02660287,0.30111122,0.3807312,0.27548146,0.5152582,-1.0428163,0.15782778,-0.4853617,0.26293224,-0.14378029,0.40722933,-0.5009273,0.55219847,0.1590791,-0.7819558,-0.0538273,0.00893654,-0.00283629,0.01358413,-0.63386065,-0.3190938,-0.204045,-0.08567032,-0.03255079,-0.45189056,0.28716066,-0.01769585,0.14898609,0.3996744,0.27093256,0.3473847,0.38096896,0.6135137,-0.254643,0.10702576,-0.657878,0.03223398,-0.12702613,-0.14168389,-0.3429342,0.19405077,-0.93699163,0.68494034,-0.48206052,-0.5542429,0.30246055,0.39754495,0.1112591,0.46350956,-0.32125545,-0.13529135,-0.00992841,-0.20913184,-0.5068834,0.8209288,-0.65059185,1.0294199,-0.1559572,-0.36477998,0.12719521,0.2333346,-0.01661515,0.12520139,-0.54308057,-0.04170252,0.14479989,0.47502548,0.09599585,-0.21922937],[0.01445117,-0.12675014,-0.13805974,0.2777157,-0.95064396,0.00699416,0.1619394,0.16445789,-0.03365368,-0.97039497,-0.11659475,-0.00318575,0.81100404,-1.4883456,-0.40900123,0.22117944,0.045945,0.1718883,0.5453937,0.66447395,0.25333816,-0.55180955,0.58817136,-0.02163359,-0.16115291,0.6243279,-0.10407471,0.43291637,0.11319701,-0.15369268,0.5174125,-0.69925195,-0.24488053,-0.38947627,0.40813226,0.17872861,-0.45989737,0.40293217,0.03011597,-0.76624477,-0.56694746,-0.08598723,-0.26678845,0.13490552,0.31882602,0.2278684,0.06083024,-0.02644216,0.08015946,0.3850558,-0.32056934,-0.3880878,-0.27601597,-1.2979165,-0.9411659,0.2502023,-0.5175512,0.37895694,-0.17787951,0.23933564,0.6040211,-0.79427683,0.03885401,-0.652147,0.64398694,0.12591666,-0.03166592,-0.69342005,0.14487419,-0.14628063,0.05360767,0.59853524,-0.16971977,-0.21360607,-0.3458962,-0.54998696,0.02615485,-0.24903944,-0.5916869,-0.8390574,-0.08456518,-0.3034689,-0.07725807,-0.4951332,0.44803765,-0.15927398,-0.3432419,-0.35723114,0.6356297,-0.11924064,-0.62835574,0.33588856,-0.3960686,-0.22190444,-0.37543428,-0.40923634,0.3307525,0.60253376,0.02586345,-0.02517697,-0.0638175,-0.02677954,0.43080842,-1.3572469,0.47768608,-0.10794237,-0.13194247,-1.03647,0.4353139,0.44521978,-0.12154906,0.2991226,-0.09012983,-0.00168282,-0.50779414,-0.01704818,-0.13688655,0.26956913,-0.9018417,0.37688154,-0.5541491,-0.28230256,0.58019215,0.19167307,-0.3597532,-0.6239631,-0.25579044,0.27807117,-0.39300233,0.6832803,0.20287581,-0.17810524,0.6037295,-0.42777672,-0.00990786,0.40253574,-0.3354192,-0.2197229,-0.22633377,0.26526144,-0.0019179,-0.36129278,-0.10298773,0.05583423,0.22668377,-0.18234514,0.14785893,-0.00924584,-0.06629312,-0.08439872,0.704542,0.45063353,-0.6521119,0.95635575,0.06825393,0.06108443,-0.38549492,-0.33788106,-0.3520603,0.54239136,-0.7158132,0.9690901,0.17637675,-0.31275517,0.09170298,0.15617754,-0.23178554,0.31779766,-0.21448873,0.67967254,0.05830579,0.24163264,-0.31144375,0.32680595,0.8591372,0.06038913,0.04329133,0.07248941,-0.3182039,0.5792974,-0.1197045,-0.34480992,-0.0091486,0.24259469,-0.22634317,0.35978562,-0.1464534,0.61459786,-0.55199367,-0.26608554,0.23798385,-0.48495057,-0.1941042,1.3416163,-0.2637274,-0.35410926,0.4221833,-0.4820005,-0.6475868,0.48405096,-0.5451833,0.52535087,-0.10537498,0.3627932,0.08799738,0.21751937,-0.65937793,-0.21153511,0.04683818,0.595667,-0.29218107,-0.30392274,-0.30044854,0.3586889,-0.26418945,0.35213417,-0.08758255,0.80349904,-0.05553975,0.05674271,-1.0353577,-0.17412777,0.17352998,-0.11353926,-0.55661273,0.55672044,-0.52209896,-0.04877069,-0.22278306,-0.34817392,0.21318834,-0.30983353,-0.5714374,-0.9362173,-0.2850211,0.31681594,0.15335871,-0.5314808,0.4173944,-0.58955705],[-0.6498253,0.06007677,0.09285427,-0.26933232,-0.14132199,0.17584708,-0.73827165,-0.12216064,-0.6899925,0.19412304,-0.09178142,-0.06097307,-0.13634549,0.36417627,-0.1768657,-0.0796245,-0.9930123,-0.11297766,0.01766556,-0.24094301,0.24828878,0.20223697,-0.03778431,-0.18870181,0.1337189,0.536368,0.11312009,-0.34246355,0.06180895,0.50190276,0.06619025,-0.08098692,-0.18123077,0.7784674,0.25275475,0.15347171,0.3155673,0.06791998,-0.5496524,-0.4363614,0.18232138,-0.18698019,0.40354696,-0.96734595,0.35762736,0.34521848,0.3692058,0.37989324,-1.2290362,0.4337176,-1.0630125,0.13349162,-0.50327545,-0.2785151,0.4476016,-0.16555944,0.24655694,-0.13503519,0.06854116,-0.63104314,-0.2788042,0.64332056,-0.2256984,0.21015656,-0.06026256,0.10271994,0.45267376,0.00345192,-0.53520817,-0.10526454,0.40737244,-0.62245893,-0.3026848,-0.44903928,0.8028039,0.173088,-0.5340853,0.16625023,0.24655417,-0.55999625,1.0688279,-0.7896523,-1.0477937,0.46099132,-0.2734752,-0.40890464,-0.4993601,-0.05652785,-0.96167076,0.45166624,-0.54137486,-0.04575441,0.35853666,0.03270034,-0.13805,-0.11309734,0.74747956,0.002571,0.16832499,-0.4857765,0.47490424,-0.07182283,0.12720278,-0.4148444,0.46157014,-0.10730594,0.37567243,0.09312586,-0.48803958,0.0262323,-0.5569308,0.648367,0.89740056,0.16812994,0.42985857,-0.68325144,0.5360577,-0.3466313,0.6937358,-0.5535071,0.01887705,-0.06585204,-0.42018113,0.37938717,0.32909548,-0.53302354,0.95928836,-0.19099024,0.21584071,0.20020105,-0.24923566,0.34898806,0.5399952,0.01324858,-0.12028296,-0.0029869,0.3159452,0.32093033,0.39752942,-0.21944949,0.3979402,0.130775,0.2842152,0.04703502,-0.10528368,-0.39117986,-0.4841567,-0.6372263,0.39425683,-0.0699884,0.431089,0.5098271,1.0223889,-0.2876059,-0.04882018,-0.6206926,0.03699668,0.00965756,-0.25835872,0.26770633,-0.13295141,0.13458696,-0.14072426,0.3593263,-0.8160428,0.34783354,-0.31168053,-0.2517381,-0.562925,0.7682961,-0.03543932,-0.57323426,-0.5585096,0.05188967,0.5073811,0.10544945,-0.35800603,0.12305214,-0.15450945,0.01170896,-0.0773283,-0.00785163,-0.51572037,-0.2663956,0.24976756,-0.66266274,0.34945226,0.62645954,0.16746522,0.03897594,0.23615958,-0.2124291,-0.23095934,0.05849033,-0.7798685,0.697316,0.04882577,0.05619596,0.29574037,0.34087852,0.22079371,-0.23378263,-0.14735113,0.44636753,-0.48473373,-0.191248,-0.10317498,-0.3494047,0.41991776,0.5129464,0.4167755,-0.06037424,0.14155085,-0.3319296,-0.46717438,0.9023696,-0.93645984,-0.39433652,0.31944263,-0.30414614,-0.72714734,-0.114722,0.38976413,0.2556901,-0.15697746,-0.49015993,-0.01648949,0.4534162,0.5588902,0.12158687,0.39655823,0.10388163,-0.03688423,0.24781765,-0.01926948,0.44037402,-0.70886344,0.21393222,-0.10443758,-0.16304176],[-0.34618136,-0.4393031,-0.06721324,-0.7518513,-0.46120116,0.05944842,-0.20724988,0.0655499,0.17229892,0.41511562,-0.7157942,0.11138355,-0.16621591,-0.07188417,-0.16317375,0.04817881,0.44206178,-0.49085718,-0.13493355,0.21590687,0.06417062,-0.2724971,0.40197936,0.2035693,0.20295241,0.05298908,-0.07075554,0.11756195,0.6116946,0.01078772,0.1818498,-0.15906508,-0.5033477,-0.26098138,-0.01803293,0.29359415,0.17189041,0.3110761,0.24716637,0.01236698,-0.17589673,0.48852643,-0.09914906,0.329031,0.29854876,-0.44309148,0.5831771,-0.46079302,0.07042953,0.04245714,0.7202885,-0.28088605,0.5813668,-0.05208999,-0.09503441,0.16395016,0.03602014,-0.5973918,-0.27179667,0.09961411,-0.56130666,-0.03289697,0.11303274,0.2765505,0.45045677,-0.04005915,0.24523161,0.5248122,0.02578885,0.00335795,-0.15009636,-0.08811787,1.7787198,-0.58475333,0.19311582,-0.1920735,-0.46630928,0.27345225,0.3754709,0.19612193,0.5649781,-0.03652269,0.22568521,0.81428826,0.05154273,0.03788856,-0.41993633,0.22023323,-0.8435541,0.3833654,-0.3937355,0.15300842,-0.32937428,0.2801038,-0.04628654,-0.31240278,-0.13736129,-0.04114934,0.33962446,0.02421468,0.01505683,0.36516824,-0.7797227,0.27279732,0.7423651,-0.00581328,0.84279716,-0.7561814,0.02088892,-0.13628463,0.15939592,-0.00901343,0.3835033,-0.45941037,-0.04253474,0.35965118,-0.41303664,0.13099961,-0.10468411,0.05566072,-0.03307629,-0.02406518,0.07393242,0.16854398,0.27693614,0.0969215,-0.0220134,-0.30797622,-0.2771206,0.02755876,0.5142154,0.29357854,0.13874455,-0.16628246,0.07232466,-0.468232,0.63553387,-0.35414112,0.11498792,-0.12424884,0.09101295,-0.39531812,0.06934014,-0.23671323,0.39609975,-0.07397307,-0.4405969,-0.15427093,-0.13798071,0.62238324,-0.1518821,0.35061136,0.03603378,0.5808526,-0.50257564,0.47740376,0.3744595,0.1383576,0.11501539,-0.00996297,-0.5550401,0.0666849,-0.09997489,-0.2125685,0.27654684,-0.16550937,0.35061064,0.01025587,-0.18063581,0.34339675,-0.25807542,0.10560551,0.15633917,0.3437212,0.06975813,-0.04175856,-0.25147128,0.07596499,0.19726302,0.07667616,-0.05279953,0.400768,-0.47411,-0.10599977,0.17006516,0.22056592,-0.18990529,0.20877701,0.9838728,-0.92378974,-0.06547216,0.14718673,-0.24097808,0.32309523,-0.3811398,-0.5289948,0.23987466,0.38886228,0.15951636,1.4941367,0.3191843,-0.35212114,-0.24390098,-0.27706265,-0.11698003,-0.27046558,0.24068454,-0.05607617,-0.5325907,0.17357837,0.2997781,0.03991598,-0.5154292,0.0527654,0.3795594,-0.26444396,-0.26568672,0.17705886,-0.3312594,-0.22161901,0.10102493,-0.30475286,0.18544203,0.37786153,-0.01508689,-0.32647184,-0.17554887,0.3379225,-0.14024329,0.5914755,-0.6143216,0.79559004,0.17263687,-0.02173076,-0.00795027,-0.07143349,0.06729614,-0.06024534,-0.37114388,0.12127201],[-7.11096004e-02,9.65666890e-01,2.86046982e-01,3.17208230e-01,-3.01488817e-01,-8.78028035e-01,-4.35415983e-01,-2.29215413e-01,1.78831846e-01,-8.94233704e-01,-2.02823222e-01,4.27003175e-01,-7.89964795e-02,2.00150341e-01,2.61220932e-01,7.63148308e-01,-8.68162394e-01,2.52702832e-01,2.07393244e-01,2.97890276e-01,7.60549009e-02,-2.59146333e-01,-9.19967532e-01,-6.89621270e-01,-7.13273168e-01,-9.40504313e-01,-5.64554632e-01,-1.01665638e-01,-6.41916543e-02,-7.29846716e-01,-3.57748181e-01,3.24917108e-01,2.97542244e-01,-1.67412348e-02,2.84507722e-01,7.83882141e-01,5.14415681e-01,2.69592497e-02,8.12337026e-02,6.11511022e-02,-3.96068662e-01,1.79009456e-02,3.26710284e-01,1.66728646e-01,5.31264283e-02,6.54509366e-02,7.79087842e-02,1.69418871e-01,-4.20873165e-01,6.31542146e-01,-3.36700201e-01,-4.13389504e-01,4.26845193e-01,2.30699256e-01,-2.14560032e-01,4.52222191e-02,3.78538936e-01,6.92625046e-01,-5.15685558e-01,-2.62616694e-01,-6.46653533e-01,-6.19985640e-01,1.56281918e-01,1.08938634e+00,5.47525436e-02,-7.44947553e-01,9.53103080e-02,7.75217414e-01,3.95980984e-01,-4.84132282e-02,6.94625616e-01,8.94850492e-02,3.23152214e-01,-8.16643119e-01,9.83940244e-01,-3.65448594e-02,-2.81899929e-01,-2.31323659e-01,4.96573597e-01,-7.50215799e-02,-5.50575137e-01,3.76408659e-02,-7.04603316e-03,-3.11081469e-01,3.26492071e-01,-6.37418389e-01,-5.51865339e-01,7.13930875e-02,-4.49286997e-01,-1.47682577e-01,-1.25808746e-01,5.46552837e-01,-1.65094152e-01,-2.76129823e-02,3.69806796e-01,2.52900094e-01,1.61110967e-01,-8.26923847e-02,-1.41563863e-01,-1.13201149e-01,7.47015700e-02,2.74033815e-01,4.04894173e-01,3.42191637e-01,-1.87521815e-01,4.73041832e-01,-3.17914784e-01,-2.91374087e-01,-7.75402546e-01,-4.99904454e-01,2.17851520e-01,6.56134665e-01,2.98228145e-01,3.15742105e-01,-4.18970466e-01,2.91234791e-01,4.61455256e-01,-4.94853467e-01,-1.85669318e-01,-2.17352167e-01,5.48964560e-01,-9.34490919e-01,-2.78377116e-01,9.98334706e-01,2.25549880e-02,4.28162396e-01,-3.60594213e-01,3.17262918e-01,-3.32773209e-01,-3.73007879e-02,-4.64450717e-01,-2.73127615e-01,2.12906525e-01,-5.29882491e-01,-1.65870994e-01,8.03433806e-02,-5.54714262e-01,-3.91011655e-01,-4.03615743e-01,-1.47424787e-01,-8.43486488e-02,2.87477076e-01,-1.78177536e-01,-4.92315918e-01,-2.13266701e-01,-9.00857389e-01,-1.69117097e-02,2.36763909e-01,3.05231452e-01,-9.18397546e-01,-3.34219933e-01,2.95578718e-01,1.93227336e-01,-8.70668292e-01,-2.52465755e-01,5.63084543e-01,-4.52767551e-01,3.12367957e-02,-4.57491547e-01,-6.96567476e-01,-6.41952097e-01,2.97284663e-01,-1.18164338e-01,-3.54809403e-01,-1.55034795e-01,6.54832363e-01,3.52516830e-01,-3.82620454e-01,-2.62461334e-01,-7.96759367e-01,4.26637471e-01,2.72579461e-01,-2.79736131e-01,-6.53026283e-01,-7.28929117e-02,-7.99404159e-02,3.61115158e-01,3.48394215e-02,-2.89326638e-01,-7.39306688e-01,4.01115388e-01,5.68960011e-01,9.07752737e-02,7.67340600e-01,3.04835856e-01,1.23038419e-01,6.50994241e-01,-4.21152323e-01,-1.16318613e-01,-8.48746598e-01,2.87259400e-01,-5.42445600e-01,-6.57493174e-01,-7.91038513e-01,-7.52473176e-01,-1.52878150e-01,4.60076295e-02,-4.72187042e-01,2.34483212e-01,2.00856939e-01,-9.27591741e-01,7.56651461e-02,-5.09037189e-02,4.04905915e-01,-6.32821500e-01,-6.27830625e-01,3.50694120e-01,-2.02354804e-01,-3.86113644e-01,-5.48651516e-01,2.64259726e-01,-1.05021097e-01,7.37108756e-04,-2.46613801e-01,3.68545540e-02,4.43675041e-01,2.88861722e-01,-4.10050094e-01,-2.66370505e-01,1.45073012e-01,-2.71744758e-01,3.13536942e-01,5.26292324e-01,-3.21475714e-02,2.31561914e-01,-5.57975948e-01,-7.04024732e-01,1.16840988e-01,5.08620679e-01,-1.12581953e-01,-1.90121755e-01,-1.18163876e-01,-3.43175918e-01,1.52053177e-01,-2.65834518e-02,-1.00187552e+00,-4.87218976e-01,6.74551904e-01,2.69698203e-01,-4.65855747e-01],[0.00355474,-1.0078671,-0.6107782,0.3026791,0.0727677,0.20150274,-0.03242829,1.1019418,-0.29864648,-0.65921867,-0.24802345,0.8803292,-0.14454311,-0.05590534,-0.28437474,-0.77310735,0.01894744,0.00986243,-0.13529208,0.3650399,-0.15795554,0.11110939,0.11508328,-0.6632438,0.39952528,-0.03803823,-0.46372482,0.81817997,0.46847206,-0.5181625,0.50360847,-0.62649035,-0.3731338,0.08420088,0.75419164,0.0264578,-0.36197555,0.3127297,0.5994568,0.22629079,-0.07851654,0.01114479,0.40613726,-0.753736,-0.05868768,0.4762443,-0.13522315,-0.08783174,-0.83329475,-0.71596473,-0.36546576,0.25881678,0.19782554,0.5969235,-0.72873104,0.457001,-0.42281532,-0.08387008,0.4345752,0.40008622,-0.05699166,-0.31019488,0.22807729,-0.5263442,-0.5630038,-0.49011278,0.10370568,0.69852334,0.00567778,0.18937428,0.28296018,0.669141,0.31909528,0.05955027,0.7352688,0.01355051,0.4856135,0.09400517,-0.37379888,-0.02583325,-0.703242,-0.47963712,-0.29623544,-0.7526463,-0.03539301,-0.25128254,-0.31865045,-0.5279174,0.1221875,0.32022718,-0.37333286,-0.3614633,-0.10216291,0.96468085,-0.2552435,-0.34247065,-0.38238433,0.08047445,0.15210633,0.48953298,0.26926646,1.2300655,0.68978155,0.10608678,0.521025,-0.42619768,-0.7246146,0.5880965,-0.04116016,-0.27891368,-0.47077945,-0.05299601,-0.5394843,0.1280279,-0.1624989,0.11368141,-0.16800201,0.26425806,-0.4180568,-0.54194796,-0.8922244,-0.35455763,-0.49870107,0.00766416,0.09338837,-0.0532271,-0.20845753,-0.32992873,0.19045186,-0.40888348,-0.19492789,0.39437714,-1.1667427,-0.26328996,0.01588185,-0.3801701,0.15824033,-0.2570863,-0.18529835,0.10671936,-0.47091934,-0.02708628,0.09409189,-0.871442,-0.07248638,0.18152991,0.27763656,-0.11099592,-0.44323716,0.01755242,-0.7781605,-0.4472249,-0.24130215,-0.16773719,0.17279515,-0.75330615,-0.40481874,-0.7518235,-0.09011301,-0.27275398,-0.5780235,-0.08054481,0.05532183,0.6195405,0.25646153,-0.6996267,0.52780515,-0.0888771,-0.1898297,0.04146169,0.63267475,-0.04026263,0.30468398,-0.61274487,0.22770303,-0.53748155,0.4054279,0.11479646,0.01446095,-0.5401066,-0.03533863,0.62122834,0.25868988,0.05183157,-0.09533176,-0.10772865,0.7768373,-1.3081309,-0.42162427,-0.27042764,-0.29323688,-0.19181538,0.5547014,0.04986481,0.38485807,-0.2343179,0.3187248,0.23933801,0.3492221,-0.90011543,-0.25460684,-0.19939832,0.8340666,-0.79898775,0.1582715,-0.4095451,0.08620237,0.5180291,-0.30606762,-0.2924436,-0.5778752,0.146531,0.7800482,0.4636337,0.32851097,-0.7008031,0.53201526,0.5677596,0.04491658,-0.63829845,-0.33450478,-0.04070033,0.3406684,0.00257589,0.43288222,-0.81709,-0.13644528,-0.50069714,0.66211236,0.31121057,-0.16012469,0.5666697,0.4013748,-0.80366385,0.00508572,0.19518471,0.10805271,0.07351095,0.20702986,0.2553785],[-1.0718789,-0.0248968,-0.05542135,0.2865707,0.33325005,0.13122469,-0.19755892,0.4072483,-0.36523494,-0.11554562,0.14754829,-0.9711134,-0.5579229,-0.68682873,0.33899415,0.08895556,0.64473915,1.1851544,-0.13732645,-0.15037921,-0.3965203,-0.07243905,-0.02847481,0.30886963,0.7172909,-0.5790407,0.54388154,0.8037322,0.19128783,-0.48685572,-0.02182844,-1.2830138,-0.05475997,0.4907221,0.38590366,0.78233147,0.10117808,-0.25050005,0.0334893,-0.27206445,-0.05176931,-0.4133198,-0.5552602,-0.30020034,0.2175596,0.46169278,0.20745026,-0.6284474,0.46143997,0.83175987,-0.39139178,-0.6132493,-1.2541492,-0.32587156,-0.23295882,-0.6792497,-0.83022976,0.0414696,0.5237051,0.13774453,-0.22762074,-0.73251456,-0.42358282,-0.5917269,-0.6820903,0.352916,-0.21608543,-0.23268229,-1.0081397,0.7468417,-0.2182342,-0.09295104,-0.5843857,0.13972344,0.04017119,-1.5639184,-1.0847859,0.32567483,-0.3896135,-1.4812602,-0.42728153,0.36823738,0.16269524,0.26687884,-0.54770374,-0.13287568,-0.34484458,-0.686193,0.41743705,-0.3848719,0.18962274,0.06641405,-1.0314858,0.41286126,-0.08674993,-0.4617837,0.27296928,0.01776484,-0.29651067,-0.15736076,-0.39853385,-0.81025916,-0.5112206,-0.14632261,-0.4630553,-0.55999947,-0.61082786,0.27614462,0.23682196,0.20375402,0.44353628,-0.01837467,0.31506428,0.10313548,0.07054854,0.00540264,-0.07547216,-0.1827309,0.40876862,-0.04681144,-0.00410729,-0.62631804,-0.31607965,-0.85880274,-0.6588882,-0.13684388,0.326613,-0.1872062,0.17118967,-0.11473073,-0.12362844,-0.24015242,-0.8136088,-0.31960648,0.81544745,-0.35455182,0.7450153,0.1321992,0.4929037,-0.42635503,-0.74122155,-0.00323853,0.04230401,0.46552667,0.14415929,0.6159144,-0.1880098,-0.13081172,-0.2574893,0.36675555,-0.05400056,-0.47713593,0.59457195,0.38818306,0.09937543,-0.59376216,0.19874918,-0.2041857,-0.76658744,-0.40912393,0.2400049,-0.79916173,-0.47472462,0.31082124,0.5834549,0.65814847,0.51634103,0.43215382,0.5408345,-0.44310683,-0.20641315,-0.81400466,-0.79087406,-0.09842374,-0.36493623,0.30631664,0.06849811,0.05660988,-0.1112338,0.54619867,-0.2023815,0.13403855,-0.8250319,-0.07471378,-0.27233043,0.19131424,-0.10667557,0.33172837,-0.5362739,0.00728985,-0.2690261,0.32613087,-0.23288724,-0.17741993,0.53333646,-1.0136116,0.1098187,-0.15784976,0.7245514,0.9739782,-0.6434023,-0.04809142,0.04402535,0.2662194,0.86136985,0.03541866,0.28126243,-0.30391675,-0.2175095,0.34482154,-0.2700696,-0.2667814,0.338344,0.27507484,-0.21846631,0.27226183,-0.56422734,0.5449615,0.03977019,-0.04066005,0.39694327,-0.57749534,-0.07866899,0.39300308,0.05155689,-0.11610871,-0.65022486,0.05319375,-0.02844393,-0.46028185,-0.07098573,-0.82926136,0.42419043,0.7687456,-0.06458697,-0.09739595,0.42560384,-0.4901688,0.13865893,-0.04556156],[5.25918424e-01,-1.65090811e+00,1.28863417e-02,-4.28958647e-02,-3.54109943e-01,-2.66050026e-02,-1.79125026e-01,-1.97950020e-01,-2.74528563e-01,-3.67563255e-02,-2.22862601e-01,3.43650490e-01,2.80642901e-02,-8.22680071e-02,-9.53934610e-01,5.79665184e-01,8.81413519e-01,-7.89962590e-01,-5.48105419e-01,-5.76642215e-01,2.31972352e-01,6.97142303e-01,5.11574686e-01,-1.72081098e-01,-6.01584554e-01,1.30105168e-02,3.29097658e-01,-9.20149088e-02,-6.03838503e-01,3.13546926e-01,-1.19332284e-01,-5.05433261e-01,6.40490577e-02,-4.62501258e-01,4.50826526e-01,1.73740625e-01,-6.83980286e-02,3.78077179e-01,2.52025485e-01,6.32692933e-01,3.74509335e-01,-9.17843103e-01,-5.12385130e-01,-2.23967001e-01,1.53270420e-02,7.06888855e-01,-9.51039016e-01,-5.81224859e-01,-4.32933345e-02,-1.85734360e-03,1.16690598e-01,-5.70705175e-01,2.47105911e-01,9.96484328e-03,2.23883420e-01,3.91440779e-01,3.43139559e-01,3.13258380e-01,5.96297607e-02,4.17801589e-01,-8.25128973e-01,4.60147381e-01,1.19036235e-01,-2.55736053e-01,1.00753345e-01,-3.82432789e-01,-4.70112592e-01,4.81214747e-02,-3.82043213e-01,-5.09152263e-02,-1.56420320e-02,2.16185115e-03,9.61523037e-05,6.77636743e-01,3.43257070e-01,9.00927424e-01,5.01099408e-01,-2.99621522e-01,2.92065859e-01,5.85956395e-01,2.85066068e-01,2.34849289e-01,3.95672470e-01,-4.62187052e-01,3.07905488e-02,-1.43257368e+00,6.14084899e-01,1.21122904e-01,-2.36855060e-01,-3.31951976e-01,5.33755839e-01,3.99981350e-01,1.76317081e-01,-2.45484069e-01,3.57634276e-01,2.78142214e-01,-5.11269569e-01,-1.19337335e-01,2.25340158e-01,5.75215578e-01,-4.64879364e-01,2.44367540e-01,-2.12233081e-01,-7.96058655e-01,8.19148272e-02,2.04761416e-01,-5.84381521e-01,3.91995728e-01,-6.87417388e-01,4.80067939e-01,-2.86925770e-03,-5.58398843e-01,-7.29547024e-01,3.11470062e-01,-5.99281788e-01,-3.89054984e-01,2.08988190e-01,-5.50385177e-01,-8.75721872e-01,-4.29466069e-02,5.67281306e-01,5.06182797e-02,-6.76696002e-01,4.58665758e-01,7.50862360e-01,-3.27886075e-01,1.39084354e-01,-1.12095237e-01,-8.25576961e-01,-1.12767786e-01,4.02227193e-01,-3.71877283e-01,-7.95112908e-01,4.18669671e-01,-3.61456752e-01,-1.68987468e-01,-1.75887477e-02,2.04633564e-01,-3.09610456e-01,-3.40163499e-01,9.18341428e-02,2.23764688e-01,-3.23610097e-01,4.54883724e-01,-8.99508297e-02,2.20588893e-01,4.00237381e-01,-2.78925776e-01,1.38141170e-01,-1.01942372e+00,3.80815268e-01,-2.41237119e-01,-6.49943873e-02,8.28198269e-02,-5.56477010e-01,-1.06103554e-01,-1.35735825e-01,-1.06267977e+00,8.40110004e-01,1.24398209e-01,-7.59750903e-01,-7.26690769e-01,-2.54193395e-01,-1.57063916e-01,-6.39728755e-02,-5.80041647e-01,2.37745225e-01,-3.46008897e-01,-1.03030115e-01,-5.92583001e-01,-1.68920010e-01,4.78046626e-01,7.32624829e-02,-1.16217099e-01,-1.77914158e-01,-4.63370115e-01,-2.50084163e-03,4.57949072e-01,4.99930143e-01,5.33159673e-01,4.58371520e-01,7.12763518e-02,1.36688594e-02,3.85189742e-01,-3.16772282e-01,-6.93947911e-01,9.26469117e-02,7.09994376e-01,1.03381002e+00,-1.19441129e-01,5.82053006e-01,-1.53854817e-01,-2.42966339e-01,-1.95772439e-01,9.85388160e-02,8.99822786e-02,-2.00225234e-01,6.55364931e-01,-1.72780201e-01,6.22102767e-02,2.94987977e-01,-8.50484550e-01,-3.14270318e-01,-6.08540773e-01,3.61195832e-01,-1.49735346e-01,4.03811932e-02,-2.36552805e-01,-1.42937467e-01,1.66772947e-01,4.66101199e-01,-6.35524809e-01,9.14413273e-01,-6.31271064e-01,-4.58674371e-01,-9.85666215e-02,2.87546068e-01,-1.25694955e-02,1.10788450e-01,-7.76227057e-01,1.57863930e-01,-1.93952732e-02,1.54751822e-01,5.18667340e-01,-1.82133481e-01,7.18093336e-01,2.07511261e-01,-1.86288610e-01,-2.58871734e-01,3.38903606e-01,-6.11493289e-01,-6.34709656e-01,2.84983009e-01,4.58626658e-01,-1.54707521e-01,-9.58857477e-01,3.71225268e-01,7.25513622e-02,2.31307015e-01,4.63683963e-01],[0.70038366,-0.6350288,-0.03899917,0.14869067,0.566625,0.03988475,-0.08852909,1.0797513,0.3670745,-0.13040213,-0.23523454,-0.34333032,0.40872243,-0.26083392,0.34853506,0.5665218,-1.1775283,0.01894091,-0.1439851,0.08749042,0.5659417,-0.11425446,0.33999637,-0.07220516,0.43964157,0.21123536,0.43544292,0.0475041,0.24290594,-0.16381225,-0.60455346,-0.13248932,-0.3292104,0.05225421,-1.1611533,0.11188939,-0.04533878,0.5997816,0.01296685,0.5124185,0.11799272,0.12752298,0.7149865,-0.7423988,-0.2447596,-0.7375001,-0.09156042,-0.28055054,0.28071064,-0.48998266,-0.18805641,-0.11386247,0.24795055,-0.278474,-0.9527851,0.89735055,-0.5561805,0.15715824,-0.87511283,-0.5627569,-0.05710369,0.45968786,-0.13852441,0.42653435,-0.37511075,0.01816789,-0.09244955,-1.0262833,-0.24438809,0.5979799,0.09432374,-0.6380945,-0.4357924,-0.5128947,-0.20618024,-0.06816497,-0.1619189,-0.20244226,-0.8251175,-0.31272584,-0.521114,0.5482918,0.25267258,0.26032427,0.68978786,0.02970899,-0.94217426,0.08379186,0.07935946,0.35303894,-0.20384452,-1.117622,-0.10610255,0.5414744,-0.01181727,-0.22226107,0.25063738,0.19029893,0.25718692,-0.4006615,0.30812162,0.55422217,-0.09652353,0.68878263,0.31523168,-0.25889745,0.057105,-0.19733255,0.43614915,-0.43065095,-1.0029787,-0.60123914,-0.8941431,-0.1228777,0.31749395,-0.15085535,0.22158706,-0.01668116,0.17606193,0.25019524,0.0581703,-0.11577237,-0.9357378,0.71274644,0.32423154,0.27522275,-0.9268894,0.05161345,0.25462118,0.24402855,-0.18699533,-0.00795299,-0.18929511,-0.06310537,-0.16075511,-0.34249932,-0.4947426,-0.04103883,-0.45166492,-0.23979351,0.4870787,0.17770831,-0.29340655,0.32277107,-0.08105288,-0.07265873,0.01885788,0.286858,-0.05098052,-0.17641948,-0.38950798,-0.17753682,0.42873892,-0.14283507,0.00343518,0.1985884,-0.07177651,-0.1155313,0.6488612,-1.1306162,0.22023354,0.12989078,0.07849176,0.17139947,0.2768089,-0.12279627,0.30988836,-0.25499058,-0.03603679,0.4136688,0.08402698,-0.39064798,0.7244381,-0.79279035,-0.16997206,0.5791626,-0.2983375,-0.44442534,0.27316567,-0.12676302,0.05731256,0.02659763,-0.49579838,-0.22134335,-0.05663482,-0.35362557,-0.03018502,-0.541199,0.28836906,-0.24680647,0.5938375,0.3702846,0.0862446,0.3590628,-0.36711887,-1.5149465,-0.06370074,0.24840231,0.02675076,-0.27964938,0.31295094,0.30821627,-0.08767594,-0.48636225,-0.5948403,-0.01539595,-0.45222473,0.01131386,0.38653284,-0.25765532,-0.12853548,0.24856123,0.2574169,-0.3500036,0.19834985,0.46951053,0.26601133,0.2949769,-0.4776277,0.14545436,0.47475645,0.46724892,-0.34051195,-0.00750751,0.21732308,-0.15806533,-0.16002715,-0.38856375,-0.36101264,-0.43171567,0.5193616,-0.58890957,-0.74153477,0.03412588,0.51994145,-0.4498847,-0.2017318,0.64035606,-0.180715,0.61252105],[3.07758003e-01,-2.69438535e-01,-1.16851039e-01,-9.95424017e-02,4.23036903e-01,7.21060708e-02,1.10536359e-01,-1.05592730e-02,-5.21255851e-01,2.35472530e-01,7.19122231e-01,-5.36074758e-01,-5.48363805e-01,2.84844309e-01,3.31661314e-01,6.42968893e-01,-2.10333243e-01,2.15368420e-01,-6.91621184e-01,1.99172944e-01,8.60621314e-03,-1.79330230e-01,5.09282172e-01,-3.89810264e-01,-9.16640088e-02,3.31306070e-01,-3.57452393e-01,-3.93263139e-02,-3.82698208e-01,-4.99749929e-02,-1.47677615e-01,-9.64285210e-02,-3.63820523e-01,3.44638973e-01,4.14099157e-01,-3.15103412e-01,-1.03939444e-01,-3.51224422e-01,-2.65866965e-01,1.21891215e-01,1.83869809e-01,1.06293046e+00,-1.97667833e-02,1.63861606e-02,4.07006708e-04,9.66829002e-01,5.58437169e-01,-5.24929501e-02,-1.14108205e+00,-3.99758555e-02,7.93900669e-01,-2.67340958e-01,-1.01954114e+00,6.77555129e-02,-4.32307065e-01,-1.99309528e-01,-1.25681981e-01,1.10856541e-01,3.67543072e-01,-5.23439944e-01,-2.95753449e-01,-3.11706275e-01,7.66096115e-02,6.22479692e-02,-5.56335188e-02,1.43432021e-01,-1.09399930e-01,1.10583179e-01,5.63114174e-02,-3.00000608e-01,-9.66939330e-02,4.26705062e-01,9.30635408e-02,-2.37274721e-01,7.35513628e-01,-6.74405992e-01,-5.03554344e-01,-1.59864813e-01,-2.67115161e-02,-6.54705882e-01,-6.02274895e-01,2.61907458e-01,-4.10290718e-01,1.13226222e-02,-1.16900277e+00,2.31149912e-01,-3.72143745e-01,7.90712535e-01,5.28242961e-02,-1.84792548e-01,2.23626673e-01,5.71514726e-01,-1.42038018e-01,-3.77173036e-01,3.61160794e-03,-1.71484966e-02,7.23157644e-01,-2.34195143e-01,4.41228271e-01,5.37932813e-01,2.31310120e-03,1.02516733e-01,-4.44999754e-01,1.22417584e-01,6.39114320e-01,-3.80495280e-01,-8.72820914e-01,-5.80188558e-02,-5.78597069e-01,1.05474341e+00,1.13973454e-01,4.64643806e-01,2.43722945e-01,-6.50875121e-02,-2.95002609e-01,-3.53276404e-03,-8.39108229e-02,-1.36671945e-01,-3.52287620e-01,-4.74940479e-01,1.99551940e-01,1.15773693e-01,-7.22975612e-01,5.36655486e-01,-1.91745654e-01,-9.55491979e-03,4.41837497e-02,-4.94284391e-01,5.62333524e-01,-4.18037653e-01,6.63531572e-02,-6.97871447e-02,4.34003800e-01,-2.68296182e-01,3.08973402e-01,7.63386339e-02,3.17124039e-01,-6.77544773e-01,-4.25466955e-01,5.44218421e-01,-3.83925706e-01,-1.73122466e-01,4.04531717e-01,-3.12577516e-01,2.46430144e-01,1.09274425e-01,7.20462382e-01,-6.96639001e-01,3.98065984e-01,1.13356516e-01,-9.62613076e-02,-2.80731916e-01,-4.01289999e-01,1.87255368e-01,-4.23888147e-01,-2.53124297e-01,2.38306988e-02,-1.79447676e-03,2.83749383e-02,-1.72153618e-02,-4.12077188e-01,5.87418914e-01,8.27165902e-01,-1.69890314e-01,-5.12096584e-02,-2.87638921e-02,-2.30041102e-01,-7.38727331e-01,-5.48925936e-01,5.70175827e-01,1.49152532e-01,-1.10251355e+00,2.75043815e-01,-1.11286663e-01,-5.81599891e-01,-1.65154338e-01,1.26349255e-01,-7.83810079e-01,-2.87043929e-01,-4.21538383e-01,2.69143820e-01,-3.94210011e-01,2.18690024e-03,3.47029835e-01,3.38905126e-01,1.29734129e-01,9.86819863e-02,-6.74782842e-02,6.85150176e-02,-8.03016484e-01,2.80586313e-02,1.57662287e-01,-1.38298497e-01,-8.01505625e-01,-1.05503537e-01,3.22795451e-01,-1.68924540e-01,5.66362262e-01,3.93008143e-01,-7.33243078e-02,6.73229456e-01,-4.65982035e-02,-1.96494043e-01,-1.59299895e-01,-1.21828592e+00,-8.52558687e-02,4.29790974e-01,5.83506465e-01,-3.38467717e-01,8.54966760e-01,-5.45825422e-01,4.55886841e-01,3.03798784e-02,1.57637015e-01,-3.13009292e-01,2.58047760e-01,2.56129324e-01,-1.49173737e-01,9.39413235e-02,4.24806207e-01,-1.80135489e-01,1.43800646e-01,-1.01845980e+00,-1.45988658e-01,-2.73674041e-01,-8.14267620e-02,-1.21817561e-02,7.52168596e-02,7.74110317e-01,2.05283776e-01,4.75345194e-01,-1.80885911e-01,-3.61560017e-01,5.09071127e-02,-4.91636574e-01,4.41423565e-01,-7.14972436e-01,-2.09829807e-01,1.81855768e-01,2.42484540e-01],[-0.6435241,-0.02184138,-0.00880781,0.501218,-0.0347387,-0.11046037,-0.21391383,-0.00187542,-0.45024747,-0.07820296,-0.04619361,-0.6241487,-0.19371644,0.42925826,-0.21705906,-0.17675114,0.5280306,0.49169365,-0.2184518,-0.2494875,-0.17221169,-0.13380067,-0.18591712,0.21542549,-1.8324382,0.69075805,0.53525674,0.10221055,-0.8197895,0.42680305,-0.6550329,0.36919352,0.47822773,-0.4383052,0.16006477,0.25349817,-1.1289176,-0.8395001,0.2726371,-0.27991256,0.05159877,-0.4711948,-0.37033734,-0.558635,0.22472858,-0.01506492,0.3911913,-0.04471295,0.00591254,-0.4796725,-0.4170133,0.0348717,-0.14257151,0.47516772,0.19122559,0.26044866,-0.23529902,0.95374775,-0.12192988,0.14786178,-1.1635213,0.12279633,-0.40567788,0.18083806,-1.0295182,-0.02136436,-0.01884454,-0.30987126,-0.13547915,0.61185545,-0.6101787,-0.14130901,0.02106304,-0.07559729,0.6905515,0.31150284,-0.12340367,-0.2009203,-0.02668972,-0.34282026,-0.02901055,-0.92777145,0.2814488,-0.78909266,-0.95809543,-0.04552374,0.28090414,0.3100262,0.39844328,-0.2804652,0.47793588,-0.31049588,-0.33244818,-0.17646445,-0.25297186,0.20074806,0.4681374,-0.84368724,0.3541991,0.25416198,0.16091055,0.11195227,0.43203998,0.10779862,0.28104272,-0.6333006,0.48989642,0.08432271,0.4973369,0.19355708,-0.12075332,-0.4689068,0.03583632,0.5657287,0.16313045,0.2028348,-0.21440525,-0.07648316,-0.99834263,0.551492,-0.04407752,-0.2503608,-0.3975918,-0.29439744,-0.8437172,-0.42616364,-0.24734308,-0.3043739,-1.092499,0.10650236,-0.12680621,0.3179842,-1.5150535,0.4964909,-0.21797363,0.27660167,0.20985636,0.29122123,0.67712057,0.3715196,0.12571378,-0.28065184,-1.0809437,0.01717362,0.5925127,-0.3931568,-0.05050048,0.04191379,0.40586942,-0.52909636,-0.19408377,-0.04781813,-0.20686679,-0.19788574,-0.51043963,-0.07906103,-0.3749813,0.19637026,0.16687687,-0.46099126,-0.3297132,0.11621439,0.4083989,0.01803676,-0.5166658,0.03983613,-0.6505857,-0.03001454,0.4669514,0.05046121,0.56469035,-0.51809645,-0.56794703,0.26323313,0.09755585,0.42361477,0.03466637,-0.36271378,-0.4569433,-0.34745726,0.4224273,0.07913323,0.21969488,0.03823264,0.03530044,-0.10697988,0.34158123,-0.06430018,0.02145749,-0.36170432,-0.4295301,-0.18293133,-0.04885281,-0.3932072,-0.40232995,0.13333242,0.31533417,-0.23644939,-0.37885615,-0.16377117,-0.15715578,0.5009204,0.7176127,0.13712887,-0.46347955,-1.2109805,0.7003459,-1.2429436,0.14096834,0.1527205,0.65049386,0.24396779,0.17338017,0.22380564,0.28998718,-0.42089736,-0.7175778,-0.4508744,-0.23359635,-0.7090802,0.39426202,-0.85464853,0.49646538,-0.0586777,0.5854002,-0.43169922,-0.11044621,0.5374964,0.44557297,-0.390038,0.04856591,0.67078334,0.14026797,0.20392746,0.27541134,0.4876149,0.5064735,0.04627209,0.2196384,-0.2001637],[0.45155022,-0.46850935,0.45200744,0.15870076,-0.25908667,0.90331346,0.10724907,0.21197735,0.04258971,-0.72238725,-0.6685904,-0.28162223,-0.5009034,0.7221013,0.06124348,0.22027819,0.26202953,-0.4450143,-0.6597949,0.21058919,-0.34970063,-0.81189185,0.44424742,-0.08758783,-0.742548,0.52830714,-0.16645944,0.20152877,-0.35856602,-0.22004542,0.38423002,0.02512021,-0.4037611,-0.23985226,-0.5324347,0.16395321,-0.15682939,0.34253645,-0.49788055,0.23875159,0.04836584,-0.08779097,-0.21278825,-0.51030475,0.5048934,0.22628897,0.69486284,0.62341034,-0.1907722,-0.02057343,0.19625431,0.39993536,-0.20555791,-0.05061188,0.27941173,-0.34173462,0.4230595,-0.15216047,-0.02086984,0.29417354,0.5580832,-0.1219852,0.54625356,-0.15767135,-0.7588972,-0.32584658,-0.46484,-0.02218377,0.11334842,-0.05406161,0.30701914,-0.35402578,-0.5205433,-0.5129457,-0.29284784,-0.38837737,0.31025362,-0.71329355,0.04268172,-0.0017594,0.09791261,0.16440731,0.16320503,-0.4031973,0.4441695,-0.17810535,0.31316835,-0.0509893,-1.00615,-0.35908034,0.04838475,-0.4918827,-0.49226993,-0.29138163,0.15321048,0.37600052,0.41438067,-0.14106444,0.6406474,-0.11341231,0.07934295,-0.6832102,0.89568955,0.34696925,0.28963327,-0.17611171,0.18440327,-0.5515435,0.02024204,0.4017379,0.9702028,0.1014941,0.14547837,-0.38558128,-1.0718057,-0.08053419,0.55682266,-0.6281633,0.14436254,0.77599096,0.03208811,-0.09973773,-0.10397433,0.57316273,-0.01703059,-0.18568565,-0.07003629,0.4518249,0.57242787,-0.90899676,-0.01208421,-1.7402622,0.16768481,0.26075357,-0.11354756,0.23044138,0.41282836,-0.43171826,0.2391216,-0.12760186,-0.46146035,-0.35875157,0.09060488,0.18515629,0.25425664,-0.1182743,-0.0284726,-0.34525362,0.05289564,0.04775165,-0.3643217,0.09477754,0.52112204,0.252423,-0.56468934,0.10647319,0.15594763,0.2515674,0.18287118,0.42316464,0.28749713,0.4793487,0.55444235,0.31815198,0.26325262,-0.58913624,0.05028936,0.2208833,-1.1909245,0.30598953,-0.71333647,0.21137851,0.44564483,0.05899494,-0.8961255,0.28032625,-0.53952664,-0.979833,0.4303047,-0.555794,0.30051017,0.4191346,-0.470652,0.16367699,-0.38592634,-0.2950805,0.11397406,-0.20757392,0.16553095,-0.51118267,-0.6920528,-0.13566211,-0.0764192,0.56414205,-0.8095478,0.26074928,-0.29677048,0.45178708,0.32968426,0.38840538,0.3497992,-0.24572223,0.39563033,-0.7972785,0.5763935,0.23862642,-0.01035583,-0.7726072,0.07658594,0.10134931,-0.09852513,0.36706483,0.15220857,0.541421,0.28441188,-0.7003914,-0.85988355,-0.02513716,0.21058609,-0.11483936,0.25282726,-0.43207306,0.1798746,-0.29115397,0.2674412,0.3375315,-0.30719686,-0.03800884,0.11277213,0.4210809,-0.24612294,0.09179144,-0.11388497,-0.39059308,0.5773981,0.1629549,-0.10516547,-0.77998435,0.09445962,0.45819435],[-1.12112269e-01,2.28820696e-01,-1.40817210e-01,9.19580996e-01,-8.90264735e-02,4.33843583e-01,-3.68222982e-01,-6.44787908e-01,2.61456836e-02,3.40434551e-01,5.77911198e-01,2.94417758e-02,2.86540568e-01,-3.21913093e-01,6.44298732e-01,3.66640329e-01,5.66329181e-01,4.37689602e-01,3.70352268e-01,1.79764241e-01,6.24096282e-02,7.95758329e-03,-1.58305377e-01,8.50350931e-02,8.35598528e-01,4.56473708e-01,-8.74140710e-02,-5.20364165e-01,6.15309775e-01,8.03614408e-03,3.54113698e-01,-3.24386537e-01,-3.59179169e-01,1.05003905e+00,-6.37332052e-02,-5.91034889e-01,7.74174988e-01,-2.23355189e-01,4.84802723e-02,3.29792708e-01,-3.48879427e-01,-1.17129222e-01,-1.49975702e-01,-7.94297099e-01,-4.65403110e-01,-4.89406675e-01,6.87796235e-01,-1.84251424e-02,-1.14742589e+00,-1.00687695e+00,-4.28177983e-01,-4.40190017e-01,-7.16570765e-02,1.97236821e-01,-9.40552354e-01,-6.30858913e-02,9.91431177e-02,-2.09205166e-01,-5.37747145e-01,-9.32571888e-01,1.64746135e-01,-6.35359406e-01,-1.08774453e-01,-2.87089800e-03,4.07327831e-01,1.09810188e-01,1.47154212e-01,-3.87218803e-01,1.43524498e-01,5.70895560e-02,-1.08853328e+00,2.43709654e-01,4.74981636e-01,-9.40746292e-02,9.46319997e-02,4.94626850e-01,1.18730731e-01,-6.09495103e-01,-1.07363135e-01,-9.10292044e-02,-1.34448552e+00,-1.29841506e+00,-2.29907155e-01,1.83133736e-01,-1.01278700e-01,1.05653667e+00,-9.58653390e-02,-7.49655783e-01,8.40078890e-02,-3.09933335e-01,9.44596827e-02,6.15872741e-01,-1.29758865e-01,-2.01805666e-01,6.42881766e-02,-2.22992539e-01,5.50250821e-02,-5.45290448e-02,-8.33717227e-01,-4.86930490e-01,3.02661240e-01,-1.37024522e-01,6.46218479e-01,6.00866556e-01,-1.67180210e-01,-4.15462881e-01,1.05246782e+00,-7.70962298e-01,-3.39934170e-01,1.71912551e-01,5.06869256e-01,3.37248474e-01,3.59247550e-02,5.44436313e-02,-9.65241194e-02,4.59111959e-01,-5.31389154e-02,-6.61123872e-01,-4.93810952e-01,4.27434057e-01,-1.89922139e-01,-5.94177127e-01,6.07455313e-01,-8.87489855e-01,1.02033257e+00,5.06188035e-01,4.47832137e-01,-5.44026978e-02,-3.34780484e-01,-8.47905159e-01,2.59450316e-01,-7.45285630e-01,-1.21662326e-01,-2.49378428e-01,3.37744802e-01,-8.30631412e-04,-5.64922512e-01,5.83940327e-01,5.61864316e-01,1.92726091e-01,4.45715040e-01,5.22366583e-01,1.23106219e-01,-2.36569196e-01,-1.54855698e-01,-9.43287238e-02,2.54638225e-01,3.09930325e-01,5.24904251e-01,5.85577905e-01,7.40769982e-01,4.40759182e-01,1.63538381e-01,2.18337607e-02,8.55109692e-01,3.44253063e-01,8.76878500e-02,-9.62944999e-02,-2.08633780e-01,2.42957830e-01,1.72369871e-02,3.91569316e-01,3.14180821e-01,2.40360424e-01,-1.35259956e-01,1.58532009e-01,3.48928064e-01,5.40430192e-03,-5.12398124e-01,-4.28870022e-01,8.09365883e-02,-1.33097649e-01,-2.71333069e-01,-1.59344792e-01,1.56919375e-01,6.63456857e-01,2.47806609e-01,1.35588413e-02,-3.24014276e-01,-1.12418130e-01,-9.74209487e-01,-4.83778626e-01,5.65264106e-01,2.60017127e-01,-5.07675745e-02,-1.10359706e-01,-1.10359795e-01,-4.32237625e-01,-3.84515792e-01,4.70008403e-01,-9.32597041e-01,-1.79094687e-01,-3.39058399e-01,-1.63744107e-01,3.11589502e-02,2.44846195e-02,1.82890028e-01,-6.96096480e-01,1.94856763e-01,-9.54127192e-01,-2.72549957e-01,-3.70764792e-01,-4.83388752e-02,-9.42397416e-01,-4.15629983e-01,1.74153864e-01,-1.38001367e-01,4.59440738e-01,-5.01897335e-01,-2.66887724e-01,-5.78605354e-01,2.95622230e-01,3.50083858e-01,-3.48202288e-01,3.22536439e-01,4.08372320e-02,-2.80261636e-01,-6.78409517e-01,3.00758481e-01,-8.14916372e-01,-4.45817113e-02,-3.90620172e-01,3.01434875e-01,1.69192366e-02,5.94528317e-01,-8.80409718e-01,-3.76130670e-01,-8.59083235e-01,-2.49628440e-01,-8.91755819e-02,1.87140536e-02,-2.25468606e-01,2.66528130e-01,-1.97820425e-01,1.89420670e-01,-2.43787125e-01,1.42229676e-01,9.77604449e-01,2.80743785e-04,-4.01435137e-01],[3.72288972e-01,6.14045143e-01,1.76394597e-01,-2.87969083e-01,2.13907361e-01,5.74786603e-01,2.10605606e-01,-2.93206722e-01,4.32091027e-01,7.46834815e-01,-8.29251632e-02,6.34235024e-01,6.82590067e-01,-3.89863640e-01,4.13804442e-01,-7.49498531e-02,-2.01244771e-01,9.83142257e-02,-3.54042388e-02,1.36796027e-01,-4.71049428e-01,-3.10288258e-02,-1.34287290e-02,3.54707807e-01,8.36532652e-01,-6.92953020e-02,-1.52885675e-01,-4.73145060e-02,3.47710729e-01,-2.88895249e-01,1.18307449e-01,-8.88881534e-02,5.57022281e-02,-3.84241223e-01,-4.66315866e-01,-4.02642012e-01,-5.14700592e-01,-1.69218197e-01,1.17322497e-01,-2.88638085e-01,-4.64099526e-01,7.42146820e-02,7.36322999e-01,6.06857896e-01,-3.58677506e-01,4.38211650e-01,3.05200934e-01,2.75635701e-02,7.94676691e-02,1.69121802e-01,-4.15053040e-01,-2.92803437e-01,-7.04690754e-01,9.43624139e-01,3.29492658e-01,-2.03692526e-01,5.38901925e-01,4.67497297e-02,2.23883703e-01,1.02284499e-01,1.90633029e-01,2.80802399e-01,-8.68633681e-04,2.08721533e-01,5.68460643e-01,6.83129370e-01,-5.96161664e-01,1.95635572e-01,-2.15974540e-01,4.68237162e-01,4.37857538e-01,2.93466836e-01,-7.07087100e-01,2.40363881e-01,-1.75602347e-01,5.01797140e-01,-3.98008019e-01,-3.44568223e-01,4.15464103e-01,-1.12057936e+00,6.77355975e-02,-8.28905255e-02,3.92819894e-03,-1.30837495e-02,-3.49549204e-01,8.00728619e-01,-3.79068814e-02,-5.39326146e-02,-2.97075301e-01,2.36516923e-01,-7.63745368e-01,1.33749306e-01,-4.86083299e-01,-3.21863919e-01,-4.52288359e-01,1.55295715e-01,5.36417603e-01,-8.73942673e-01,2.59124696e-01,7.33468056e-01,-4.10683692e-01,1.08392544e-01,-3.31814736e-01,-8.05249214e-02,-1.30159855e-01,-4.90840495e-01,-1.76419497e-01,4.25927453e-02,-7.92723820e-02,3.14487726e-04,3.47054988e-01,5.32731712e-01,-2.95753837e-01,-4.21910554e-01,6.09516084e-01,4.40973751e-02,-6.24652326e-01,3.49481583e-01,-9.25588757e-02,-1.08069789e+00,-6.22630358e-01,-1.27842814e-01,8.06060061e-02,-3.75971586e-01,-2.86658376e-01,-2.69729376e-01,-5.02536237e-01,-3.23434860e-01,-5.17282426e-01,-5.01734912e-01,-5.81351519e-01,-3.95034611e-01,4.21380252e-01,-4.26373273e-01,6.93787262e-02,1.17873944e-01,1.05461335e+00,-8.40886295e-01,1.10515155e-01,-3.86375576e-01,4.13399152e-02,-1.50679816e-02,-2.75807738e-01,-3.10793340e-01,-1.22913523e-02,6.38074696e-01,1.34321928e-01,-8.39058995e-01,-9.40180242e-01,3.25604022e-01,3.03635001e-01,1.79590508e-01,5.65124691e-01,2.25010037e-01,4.71895039e-01,3.40282023e-01,-2.63074428e-01,3.26339304e-01,-1.67436935e-02,-4.66847457e-02,2.05316857e-01,-7.66480863e-01,-3.10798157e-02,-1.04958677e+00,5.33679090e-02,-4.86008614e-01,2.50353128e-01,3.22528899e-01,4.08595234e-01,-1.38746768e-01,-3.72932792e-01,5.17240167e-01,-3.11291009e-01,-8.81412253e-02,-9.13974866e-02,-4.36764956e-01,5.90209544e-01,3.04316729e-01,-2.69543603e-02,-2.40133584e-01,2.93754935e-01,-6.31892458e-02,1.60256550e-02,3.59674960e-01,1.37914330e-01,9.65395719e-02,4.68210019e-02,1.20879831e-02,-4.75637645e-01,1.18659101e-01,1.63059816e-01,3.27105597e-02,-7.43343413e-01,-5.98582447e-01,-1.54595688e-01,-1.30128837e+00,2.25926086e-01,-1.11852773e-01,2.81368166e-01,-5.48005998e-01,-4.96232390e-01,-5.75316429e-01,-3.64240408e-01,-1.14571728e-01,-1.20973192e-01,-3.82295310e-01,-9.72537845e-02,-2.04033881e-01,-9.97804850e-02,-1.06224251e+00,1.20374702e-01,-7.10195482e-01,4.87712622e-01,7.99815208e-02,-2.34113455e-01,3.63678902e-01,-6.98713839e-01,2.52163321e-01,2.42518410e-01,-2.16615409e-01,3.60822417e-02,-6.54911816e-01,-7.95019388e-01,-1.60795311e-03,-6.75105035e-01,4.73903030e-01,3.05840343e-01,-3.35441917e-01,5.45095205e-01,-5.42523265e-02,-1.04795635e+00,6.70108795e-01,-3.05558801e-01,4.91089858e-02,2.08594486e-01,-1.16247863e-01,-1.87666446e-01,3.81669402e-01,3.14420193e-01,-3.55037689e-01],[-2.58749694e-01,6.13512397e-01,5.35620689e-01,-1.06920436e-01,-4.18748194e-03,-6.85920473e-03,-2.37581193e-01,-1.89993784e-01,-2.25292951e-01,-2.99127817e-01,5.32799587e-02,1.28514338e-02,-7.28322864e-02,4.83885825e-01,7.06742331e-02,-1.97647244e-01,2.46202096e-01,4.96062756e-01,7.91243196e-01,-2.70180732e-01,-1.82907581e-02,-1.75205871e-01,-5.24165571e-01,-7.69219995e-02,-6.42778575e-01,-1.29847927e-02,3.60433310e-01,2.39119609e-03,4.36570555e-01,3.19983542e-01,1.44246936e-01,-2.26121128e-01,-6.62087500e-01,1.23192035e-01,-4.14045751e-01,4.58148122e-01,-2.52604544e-01,2.33935148e-01,8.84343013e-02,-6.60246313e-02,-3.33596356e-02,6.80593625e-02,-8.16387892e-01,-2.65518744e-02,7.52588153e-01,-6.26559377e-01,-1.64475426e-01,2.84635872e-01,-5.03432393e-01,-3.12344283e-01,1.21985994e-01,1.70632917e-02,5.85232675e-02,-1.25714824e-01,-1.04907465e+00,6.03345223e-02,8.68349150e-02,-4.29325737e-02,-9.83855247e-01,-4.84427601e-01,-1.24943599e-01,-3.31293911e-01,1.77420840e-01,1.16392553e-01,-2.66030937e-01,5.30711055e-01,-5.48297882e-01,1.29043907e-01,-4.31213498e-01,8.40739369e-01,3.12665291e-02,4.43387210e-01,-4.97806400e-01,-2.97449261e-01,1.64537415e-01,4.48896259e-01,-2.38186233e-02,-4.07545894e-01,3.45864415e-01,5.37465513e-01,-5.14586151e-01,4.38120931e-01,-4.80156094e-02,3.99483234e-01,1.98530808e-01,1.00901715e-01,1.84917673e-01,1.00204557e-01,6.63989842e-01,-2.97694147e-01,-4.20109898e-01,1.22817807e-01,-2.55856037e-01,-3.80905449e-01,-1.46270052e-01,2.90057391e-01,-1.31061077e-01,6.94196448e-02,5.58685027e-02,7.65078723e-01,-8.15783739e-01,1.48764968e-01,-4.96457487e-01,1.32232696e-01,-2.47659251e-01,-1.36876917e+00,7.04037189e-01,-2.36099213e-01,-1.88431457e-01,5.13515115e-01,-1.54843628e-01,2.92147130e-01,-1.48601696e-01,-4.48629260e-01,1.26762480e-01,4.82698470e-01,1.55052647e-01,7.29612947e-01,-9.95053351e-03,-1.93307504e-01,-3.99206787e-01,-6.46662831e-01,7.66840950e-02,-9.92598087e-02,-2.37615600e-01,-1.61930978e-01,-3.45193535e-01,6.22830749e-01,-6.15900904e-02,1.00718416e-01,2.22374476e-03,7.60419145e-02,-4.65937734e-01,5.51861107e-01,-8.35042417e-01,-2.48362962e-03,-3.80336553e-01,4.65212651e-02,-2.86469072e-01,-3.26258957e-01,2.02133227e-02,4.84446287e-01,-4.10491467e-01,-2.47923518e-03,5.96266985e-01,-3.74320805e-01,3.87556016e-01,-6.65296137e-01,-1.02192807e+00,7.50692666e-01,-8.89595002e-02,-2.37366274e-01,-1.60171404e-01,-1.76348463e-01,-5.73986650e-01,-4.79145974e-01,2.27723315e-01,-3.32725883e-01,1.62080824e-02,-4.00852650e-01,-2.37526193e-01,-3.39928985e-01,-8.16269875e-01,2.06085742e-01,1.55815232e+00,-6.18904650e-01,4.02088523e-01,-5.54705143e-01,-3.35284583e-02,-6.27223179e-02,-6.45650506e-01,-4.09350932e-01,2.93426335e-01,-5.04200757e-01,-2.26448804e-01,-3.81979555e-01,-3.06466639e-01,3.14708650e-01,-2.76378483e-01,-5.16097426e-01,-2.57655591e-01,-6.44662976e-02,3.63040239e-01,2.88213283e-01,1.66323811e-01,5.71887791e-01,3.50232273e-01,1.34830624e-01,4.32523340e-01,-3.46424431e-01,2.91132301e-01,2.53928006e-01,1.07927181e-01,3.22224766e-01,-8.82723391e-01,4.22223896e-01,-1.58642277e-01,3.49889457e-01,-9.27006602e-01,-4.32104051e-01,5.53511791e-02,8.44055712e-02,-1.44549644e+00,-2.61872023e-01,-3.96527275e-02,6.28333241e-02,2.60682851e-01,4.37496990e-01,1.86390191e-01,-1.66686520e-01,-1.88587740e-01,4.62290287e-01,-4.81865227e-01,-2.73928165e-01,3.11284989e-01,5.95314614e-02,-4.60138589e-01,-1.37702912e-01,4.30260986e-01,1.33087263e-01,-9.16224380e-04,-4.76712435e-01,-4.38864857e-01,1.90235570e-01,4.79893923e-01,1.20331794e-01,-3.93647641e-01,2.02371806e-01,-7.34767795e-01,4.52887602e-02,-5.44396162e-01,-2.66311288e-01,-2.91899413e-01,-1.41786665e-01,2.10060745e-01,-2.54860014e-01,-3.08348984e-01,2.45838299e-01,5.54588854e-01,-2.35528186e-01],[2.88451032e-04,1.83955580e-01,3.08969766e-01,-3.53826582e-01,-4.74806845e-01,-2.76252300e-01,2.55576551e-01,5.48188295e-03,-2.26249576e-01,-3.05493712e-01,1.42996460e-01,-8.49338919e-02,-4.79669750e-01,-3.86856705e-01,2.92401165e-01,-4.92748395e-02,-3.31433713e-01,-9.74124074e-02,-7.29666233e-01,-1.00983471e-01,-5.83965302e-01,9.55309048e-02,-6.77731454e-01,9.97340679e-01,-8.28986347e-01,5.11966586e-01,-1.41881263e+00,2.78486490e-01,4.83193159e-01,2.04551116e-01,-1.60369650e-01,-7.63560116e-01,-1.19797878e-01,1.65400445e-01,2.22123906e-01,6.08367212e-02,-4.33937926e-03,-1.82631731e-01,4.55980599e-01,3.31479520e-01,5.17343700e-01,6.14937246e-01,6.67084157e-01,-2.51131237e-01,7.59313822e-01,2.89937764e-01,-1.12466432e-01,-4.87043798e-01,5.81128784e-02,-1.39250770e-01,5.00652552e-01,-4.18365180e-01,-1.36725143e-01,2.59620190e-01,2.98916876e-01,3.54833491e-02,5.42109787e-01,-1.16606617e+00,-9.83314216e-02,5.76081812e-01,2.46440217e-01,3.57991964e-01,4.88896161e-01,-7.78776586e-01,2.18189120e-01,-4.68379557e-01,-5.28881967e-01,2.98682004e-01,1.16609275e-01,3.99844259e-01,6.78050399e-01,-7.18344569e-01,-3.06576431e-01,-1.55052423e-01,-3.45395282e-02,6.49192691e-01,-1.19844019e-01,4.05274957e-01,2.87697792e-01,3.48009378e-01,-2.81405766e-02,-2.89757103e-01,-3.84685993e-02,-2.52946496e-01,-1.43525541e-01,1.12051200e-02,-2.48096079e-01,8.25325191e-01,-3.70382100e-01,3.25713843e-01,3.73559982e-01,-4.34129179e-01,3.49808812e-01,-2.74143815e-01,6.72955394e-01,-2.44814515e-01,-1.28117144e-01,-1.42614869e-02,4.67466146e-01,-1.47629991e-01,3.15213874e-02,-6.81243598e-01,4.18088555e-01,-7.43265748e-01,-1.68123037e-01,-2.65438527e-01,-4.11769450e-01,3.24794620e-01,3.95070501e-02,1.74626764e-02,2.92796195e-01,-9.53100324e-01,1.86656296e-01,-1.15477256e-01,-6.72490299e-01,2.90801167e-01,3.01407874e-01,1.93708226e-01,-1.08857691e-01,-4.85893816e-01,3.45208138e-01,5.53505123e-01,3.04501712e-01,-3.34456265e-01,-6.31215498e-02,-1.61365867e-01,-6.60990357e-01,-3.55443388e-01,2.06695989e-01,3.68754476e-01,6.24450088e-01,-9.46753323e-02,1.15715325e-01,-6.31592348e-02,-1.18774455e-02,-1.13908887e-01,1.35009646e-01,-3.95518392e-01,-3.96072358e-01,4.29499656e-01,-6.40565813e-01,3.25097710e-01,-2.12163851e-01,-6.93955004e-01,-4.49737310e-01,-2.67068833e-01,-2.02621162e-01,1.67059585e-01,-1.31607234e-01,-2.21446138e-02,6.02218390e-01,-2.19157442e-01,3.44986439e-01,8.82864177e-01,-2.12763578e-01,-4.28786188e-01,-4.16996658e-01,7.53933072e-01,-6.58787191e-02,-7.66663015e-01,1.89805171e-03,4.66039106e-02,-3.48639995e-01,1.00546246e-02,-1.41762011e-02,-3.96237969e-01,2.34984592e-01,-5.13654053e-01,5.05069315e-01,2.34047800e-01,-2.25489154e-01,-2.82700241e-01,1.02970421e-01,3.96956295e-01,-3.77958834e-01,8.13391745e-01,-9.02173594e-02,-1.59860060e-01,-9.86666977e-02,-8.12634647e-01,-3.68636817e-01,-2.97222972e-01,5.54161251e-01,2.01906085e-01,-5.28325796e-01,2.25753456e-01,-4.22476567e-02,5.40713906e-01,1.35339759e-02,-1.31742805e-01,4.26037401e-01,3.01628053e-01,-6.08179569e-01,2.96725452e-01,-7.14222789e-01,4.49257523e-01,4.67681050e-01,-1.02064705e+00,1.65914670e-02,-8.10813546e-01,-4.37398762e-01,2.04480141e-01,8.24701190e-02,-1.41456321e-01,-5.74692249e-01,-6.17649794e-01,-4.31090653e-01,1.27652436e-01,-2.13168979e-01,-4.30953801e-01,3.33522707e-01,-4.23167467e-01,1.95621550e-01,6.50160372e-01,6.61193430e-01,4.49047327e-01,4.51762408e-01,-1.54280707e-01,-6.04447305e-01,-1.07743633e+00,2.63763100e-01,1.08014154e+00,3.31897229e-01,8.09736364e-03,2.51739062e-02,4.60200191e-01,1.80766061e-01,-5.77058077e-01,-2.22116753e-01,-2.03196183e-01,-2.83274114e-01,2.60127932e-01,6.94016933e-01,4.38420206e-01,-6.90150082e-01,-2.25355625e-01,-3.67277451e-02,-2.46003480e-03,-4.89115924e-01,2.52461851e-01],[3.84541184e-01,7.05895066e-01,-2.09475279e-01,-9.35297385e-02,-4.87376899e-01,-3.37457359e-01,-9.55975056e-02,4.56218887e-03,3.19696844e-01,-1.13206811e-01,-1.02621056e-01,-2.27596343e-01,4.41614509e-01,-2.95940638e-01,-9.25020054e-02,6.31492632e-03,-1.93333834e-01,3.09716731e-01,3.61026198e-01,-9.73221213e-02,-6.73843920e-01,1.43406212e-01,-9.21760261e-01,1.31148130e-01,1.21518195e-01,2.51427263e-01,1.59619078e-01,-2.94570178e-01,-2.81839937e-01,5.22653401e-01,-1.42892739e-02,2.14870095e-01,4.53277141e-01,-3.29808503e-01,-1.70283437e-01,-2.54622310e-01,-8.42412654e-03,3.08823556e-01,2.62668580e-01,-1.59884803e-06,2.20040053e-01,2.25410178e-01,-3.00746895e-02,-3.28388035e-01,3.97605330e-01,-5.75953662e-01,-4.66609031e-01,-6.63455844e-01,-2.20773935e-01,1.22247770e-01,-8.90918612e-01,1.20286800e-01,-5.27998924e-01,-5.03068805e-01,-6.23887241e-01,2.21013442e-01,1.97016492e-01,-4.09007192e-01,4.65544611e-02,2.15964049e-01,-1.91573575e-01,1.66186634e-02,3.53615373e-01,-2.34590992e-01,4.67648685e-01,-1.22916128e-03,-6.12136900e-01,-2.74629146e-01,-2.05810130e-01,7.82079920e-02,-3.77947450e-01,-1.82739586e-01,4.28879082e-01,-1.29847705e-01,-7.97745109e-01,2.76532650e-01,-7.93410003e-01,-2.91791350e-01,-4.06627268e-01,4.45589304e-01,1.39802113e-01,2.43848220e-01,-1.35635123e-01,-7.19678551e-02,2.30063096e-01,3.15105617e-02,-3.81427050e-01,1.30140021e-01,6.49446070e-01,3.08326811e-01,1.33644953e-01,-9.28435564e-01,2.38551632e-01,-3.24390978e-01,3.67539138e-01,-4.75102246e-01,-1.45243434e-02,-4.20745760e-01,-4.49928939e-01,-3.52645695e-01,-2.05303252e-01,-9.56704542e-02,-1.11794734e+00,2.52894044e-01,3.26408930e-02,-4.54792380e-01,1.18395209e+00,6.48452401e-01,5.27636968e-02,-8.40614021e-01,-6.71645761e-01,-4.45313454e-01,-3.23932499e-01,1.90002412e-01,-2.49172910e-03,1.23495065e-01,5.47458351e-01,-9.70688403e-01,-4.43575978e-01,5.54170370e-01,-4.43869859e-01,-9.14060846e-02,1.17463112e-01,-6.89726233e-01,6.72615394e-02,9.72527862e-02,4.31264192e-01,-3.21209393e-02,-3.44294533e-02,2.91702747e-01,5.63413680e-01,9.33874786e-01,7.13823378e-01,2.76445210e-01,1.24417447e-01,-4.47573751e-01,1.42513976e-01,-1.30915925e-01,-4.12595943e-02,1.36454895e-01,-3.85810405e-01,-2.60593027e-01,3.79041165e-01,-7.96492994e-02,1.95168898e-01,-4.61360246e-01,3.16493779e-01,-1.76773109e-02,-9.55082662e-03,4.93466228e-01,-1.66943297e-01,3.22001129e-01,-1.28169522e-01,-9.30677056e-01,-8.27428758e-01,7.17381835e-01,3.80476058e-01,7.44143128e-02,-5.06931365e-01,-3.59210134e-01,-1.49188191e-01,5.77767789e-01,8.42689931e-01,-4.45695400e-01,3.85264829e-02,-4.77327973e-01,-3.40903610e-01,-3.89914781e-01,1.49529889e-01,2.38600615e-02,7.16974810e-02,5.60829699e-01,-2.26123780e-01,-7.16857761e-02,3.15364480e-01,-4.35841382e-02,-4.16448206e-01,-6.10849774e-03,4.87176143e-02,-6.09566212e-01,-2.76937988e-02,-5.91777563e-01,3.78255010e-01,5.91329455e-01,4.69879776e-01,1.96589362e-02,-4.95804280e-01,-7.19172120e-01,-4.09325302e-01,-4.18848962e-01,-4.09469724e-01,3.60849989e-03,1.98997214e-01,-1.42094553e-01,1.22531883e-01,2.26079091e-01,7.17763901e-02,4.56536978e-01,5.63515604e-01,4.10791755e-01,2.91296095e-01,-5.45350790e-01,1.69719920e-01,5.07621802e-02,-7.32059181e-02,2.45572850e-01,-8.52754235e-01,1.54780999e-01,-1.15024731e-01,-2.33084664e-01,-2.26019800e-01,-7.39837527e-01,-3.81463408e-01,-3.55668753e-01,-4.41637337e-01,-3.75830710e-01,1.34726867e-01,9.37921777e-02,-1.93277180e-01,-3.77994776e-01,3.26518923e-01,3.68255109e-01,3.45856398e-01,-1.66070089e-01,2.92838991e-01,-5.93890071e-01,-2.02322498e-01,2.44118348e-01,5.50277531e-01,1.18618354e-01,-5.07212877e-01,1.55412301e-01,-2.37540752e-01,-5.71911819e-02,-1.26224577e+00,5.08723259e-01,-2.38639817e-01,-5.99208117e-01,-4.37708408e-01,-2.00386107e-01],[4.44292231e-03,-1.42758768e-02,1.22267857e-01,-4.08691525e-01,-1.06039636e-01,8.12928155e-02,9.98141766e-01,-6.43466532e-01,2.78205033e-02,-1.61989912e-01,1.70290291e-01,3.66024494e-01,2.21085191e-01,-4.52073753e-01,1.42517723e-02,-2.09614560e-01,2.38273174e-01,-1.17283911e-01,4.79174763e-01,3.73321742e-01,-6.50837943e-02,2.18565226e-01,2.73538232e-01,5.91200471e-01,5.15352130e-01,-2.01582015e-01,2.07510009e-01,-1.76491022e-01,5.25532365e-01,-5.45690238e-01,2.93018907e-01,-9.37651843e-02,-3.32409531e-01,-1.83832526e-01,-7.97953010e-01,-1.00059412e-01,4.67518538e-01,-4.40970510e-01,3.80108863e-01,-3.32784772e-01,-8.78166258e-01,5.80975890e-01,-2.47695092e-02,-2.15702087e-01,-2.96714008e-01,2.59933621e-01,1.03601301e+00,3.75059426e-01,-1.49045622e+00,1.99465081e-02,-4.62555215e-02,-7.04511330e-02,1.58497304e-01,-2.76685059e-01,5.41451395e-01,1.67914987e-01,-2.98651487e-01,-1.72846075e-02,-2.78840154e-01,1.76968463e-02,-3.03440332e-01,-2.69593865e-01,-3.84289294e-01,4.91345763e-01,-6.13324940e-01,-3.34233254e-01,1.94337577e-01,-3.98740321e-01,5.61871827e-01,-3.50935936e-01,-1.32173985e-01,8.48343372e-01,-7.61854798e-02,1.32950440e-01,-9.23106968e-02,-3.75136167e-01,-4.78988200e-01,-3.48054498e-01,-2.02365860e-01,-3.60807508e-01,4.66810971e-01,-9.10775810e-02,-3.86277288e-01,-1.19927883e-01,9.28378105e-02,-5.36947727e-01,-2.98396140e-01,1.05107710e-01,-1.61335754e+00,1.25269964e-01,-1.15854360e-01,-1.54329121e-01,-3.18807065e-01,3.35958809e-01,2.88978010e-01,-7.10235238e-02,-1.19673170e-01,-1.86556410e-02,-3.92463624e-01,-1.37895867e-01,-2.36893911e-03,2.90549435e-02,-6.66580677e-01,-4.82142091e-01,4.57614154e-01,1.82338491e-01,7.62133151e-02,-3.77393961e-01,4.01242733e-01,-6.68653071e-01,5.89967780e-02,5.65947145e-02,2.60729343e-01,-5.23163676e-01,7.09814578e-02,-1.91954881e-01,-1.79590732e-01,9.88797605e-01,-1.78719357e-01,-2.91402698e-01,-7.08639994e-02,3.31363827e-01,1.44094422e-01,8.59397724e-02,-1.26014939e-02,3.43400657e-01,5.88715434e-01,-4.28118780e-02,5.27639210e-01,4.67670411e-01,5.13190210e-01,-2.90564269e-01,4.31053668e-01,9.41137820e-02,4.73619878e-01,-1.85389742e-01,6.79198444e-01,6.98978364e-01,8.68157685e-01,-6.27976894e-01,-3.34581196e-01,8.42957675e-01,-4.22452241e-01,7.00688720e-01,-1.31650490e-03,-2.75086254e-01,2.77545244e-01,9.87839326e-02,7.04681993e-01,-4.78004009e-01,-1.90391302e-01,6.81570947e-01,2.24207252e-01,6.37236774e-01,-7.16380835e-01,-5.10625362e-01,-1.63145494e-02,1.22304492e-01,-4.88420844e-01,2.58668382e-02,8.17907229e-02,-1.61719844e-01,-4.71038092e-03,1.12425536e-01,-4.24825400e-01,5.97501040e-01,-3.48695427e-01,1.31314948e-01,2.58443177e-01,-2.27677152e-01,-8.61370564e-01,-3.29097331e-01,5.39896041e-02,-9.84714273e-03,5.49127281e-01,1.70504563e-02,7.95429852e-03,1.20417923e-01,-6.57406628e-01,3.21755171e-01,-1.05136029e-01,-3.17877203e-01,-2.52371132e-01,-3.85642469e-01,1.96706370e-01,-4.78204787e-01,3.28897536e-01,-5.43391883e-01,-2.40455896e-01,-1.60935432e-01,-4.71736372e-01,1.41788304e-01,-4.91575539e-01,-8.81689310e-01,-8.15516487e-02,4.17056859e-01,-1.57118857e-01,4.27769154e-01,-3.57290395e-02,-1.37956738e-01,2.12002203e-01,-5.12379169e-01,1.75820217e-02,1.52245402e-01,-2.79708177e-01,1.03757761e-01,-1.80532545e-01,-3.65252614e-01,5.76317847e-01,-3.18030775e-01,-1.16353357e+00,-1.97281867e-01,1.24718346e-01,-8.89925808e-02,-2.44733408e-01,-3.64813358e-01,-5.56452811e-01,9.98643488e-02,1.43887967e-01,2.20881328e-01,-4.57708448e-01,-2.97723264e-01,-9.28950682e-02,-7.43431211e-01,-5.62335372e-01,2.25473940e-02,1.45018864e-02,-4.53930050e-01,-6.69047087e-02,-2.07602262e-01,1.82861406e-02,3.84825431e-02,-1.62238464e-01,1.00509977e+00,-3.41028363e-01,-3.83772850e-02,-1.48412501e-02,3.56363475e-01,5.37383378e-01,-1.84493363e-01],[-0.53354555,-0.33405888,-0.1881417,-0.32757923,0.33326063,0.03866244,0.6853255,0.00493503,-0.35098475,0.14364868,0.70158195,-0.38005832,-0.854239,0.6867321,-0.7579255,0.36725765,-0.46899626,-0.03126018,-0.37059045,0.16278176,0.33336636,0.32610017,-0.37843946,-0.09859821,-0.57207114,-0.17651826,-0.07574254,0.3135711,-0.28131908,1.1105621,0.00693388,-0.2726021,0.48245543,0.13283111,0.4481837,0.21917398,0.23107225,-0.01613761,0.2682286,-0.09875405,0.36605516,-0.62792814,-0.00260469,-1.1402855,0.08336721,0.27058396,-0.6007917,-0.18067865,-0.17875302,0.47293037,-0.08176829,0.70529866,-0.90194887,0.07063066,0.8587818,-0.29009223,0.3152544,-0.7316136,0.45263064,0.01926476,-0.24439435,0.7586599,-0.29303145,-0.22378358,-0.9278831,-0.75401425,0.34320965,-0.05279828,0.15983981,-0.48922235,-0.02865121,1.0082765,-0.14024355,0.4013698,-0.33641937,0.68339896,-0.03352943,-0.28154752,-1.1267854,0.06120994,0.19824447,-0.13773803,-0.2195911,-0.4949837,0.00373712,-0.21227881,-0.05259364,0.26405695,0.09667415,-0.15504698,-0.6394664,-0.06921858,0.53435004,0.10513996,-0.09455376,-0.5552995,0.12175231,0.5040983,0.0896789,0.14945737,0.47772598,-0.47016546,0.30686206,-0.10451948,0.17153516,0.6357669,0.32486966,0.24422012,-0.30926177,-0.54587144,-0.2164654,0.15191345,0.198211,-0.29565674,0.6792308,-0.15578377,0.7697714,-0.70587564,-0.5442567,0.29499838,0.31483606,-0.3685423,0.23968711,-0.43868563,0.2532149,-0.15396363,-0.46488038,0.22353505,0.21749549,-0.06117602,-0.44707707,-0.7126114,-0.29190728,-0.04623626,0.2526161,-0.10176871,0.05893642,0.9147804,-0.24197674,-0.43312398,-0.23620756,-1.3384926,0.49590957,0.32496464,0.29948848,-0.04561866,0.15231976,-0.37722048,-0.2691664,0.29292592,-0.54802674,-0.9334958,0.20717813,-0.33153465,0.70923537,-0.3152505,0.69805044,-0.50071037,0.19526471,0.01848547,0.21607973,-0.5071779,-0.46716136,0.17724687,-0.29477087,0.02949956,0.1882997,-0.19691704,-0.04660537,0.08409704,-0.17747721,-0.00618069,-0.30832207,-0.44428602,-0.14374198,-0.3700601,0.49364126,-0.04389046,0.42218396,-0.19071391,0.17881128,-0.2827315,0.41103533,-0.28244868,-0.37338787,-0.20808744,-0.5764306,0.46768144,0.16002338,-0.13824892,-0.2594032,-0.18857867,0.16731066,-0.27477103,0.08176093,-0.20409352,-0.48217022,0.40769687,-0.14350322,-1.1155577,0.13512598,0.03026689,0.11317756,0.07212868,0.20411955,0.51104945,-0.7651578,0.2694186,-0.32858533,0.6867186,0.23056233,1.0362519,1.1925681,-1.3719631,-0.18850987,-0.3238357,0.52664924,-0.5551928,0.35232508,0.34728885,-0.09325418,0.17606345,-0.12459856,0.02840068,0.45986035,-0.12621781,0.26326746,-0.15150748,-0.028868,-0.05001731,0.11342432,-0.5190548,0.04438277,0.31041795,-0.07000976,-0.19513391,0.29320213,0.16114374,-0.37669775,-0.1590185],[-1.44454584e-01,6.07597709e-01,-1.29570827e-01,-1.65058672e-01,-1.18936785e-01,1.75856501e-01,-4.18072075e-01,6.32763326e-01,5.27813584e-02,1.17351137e-01,4.39424030e-02,1.40112907e-01,9.35752019e-02,-8.74280930e-02,3.26715767e-01,-4.94059324e-01,-4.81066555e-01,5.20481765e-01,6.31491780e-01,-8.04647729e-02,-5.01867831e-01,-6.43589348e-02,1.48414075e-01,-2.35573098e-01,6.00262761e-01,-5.23886643e-02,-7.68657267e-01,-3.68361086e-01,-1.96457878e-01,4.76154715e-01,3.01876932e-01,9.84352753e-02,-1.63029397e+00,-6.39420450e-01,-1.00721732e-01,-7.94807002e-02,-1.07332110e-01,-5.56466123e-03,4.67437088e-01,-4.40052971e-02,-8.34337115e-01,3.71472508e-01,4.05638307e-01,4.67456907e-01,-7.05566645e-01,-3.66597883e-02,7.11698771e-01,-1.60889894e-01,-1.09859443e+00,-4.03395265e-01,-8.13471526e-02,-3.59729677e-02,-6.92284167e-01,4.67781335e-01,-2.09625512e-01,5.39039791e-01,4.91658561e-02,-1.35244057e-01,-1.24540672e-01,2.39561453e-01,-1.40214995e-01,-6.06251180e-01,-6.29940405e-02,-4.24093604e-01,5.03132820e-01,-3.28818291e-01,-3.29100847e-01,-2.99002439e-01,-3.49357456e-01,-2.68359128e-02,-4.49302077e-01,-2.86892146e-01,-5.42412281e-01,4.84751582e-01,7.59204924e-01,-1.39867112e-01,2.92700738e-01,6.08004853e-02,2.73635387e-01,2.59366363e-01,7.27955818e-01,3.35205913e-01,-1.85319781e-01,3.73433530e-01,-1.22162960e-01,4.35418725e-01,-2.65400738e-01,3.13605547e-01,-6.34792805e-01,2.01291293e-01,-9.29104686e-02,-1.81714728e-01,1.74355730e-01,-1.33444577e-01,-5.31569839e-01,-6.66186661e-02,1.18633878e-04,-7.21028566e-01,3.88845086e-01,-5.48094809e-01,4.16955769e-01,3.64862323e-01,-4.50187027e-01,-1.29058287e-01,7.32549369e-01,-1.94677129e-01,-5.79835355e-01,1.46091118e-01,3.63636166e-01,4.28972095e-01,1.34549379e-01,-2.67426878e-01,1.45139590e-01,3.62838209e-01,-1.27559841e-01,5.54051161e-01,-3.82410884e-02,9.07433629e-02,-1.67490751e-01,-5.35813253e-03,-9.68665928e-02,-3.05455476e-01,-9.25635338e-01,-1.30714273e+00,-7.43787229e-01,-3.26491863e-01,-8.09380487e-02,-1.85317338e-01,-4.25932318e-01,-1.66640133e-02,1.60504639e-01,-1.92035899e-01,3.30182821e-01,-1.36069298e-01,4.74762827e-01,-1.72865793e-01,9.12110656e-02,-3.50698054e-01,5.74171245e-01,-8.69181633e-01,3.28266412e-01,-9.66227293e-01,5.08403599e-01,3.68888736e-01,-1.32387383e-02,4.10685390e-01,-1.23585463e-01,1.86513439e-02,-1.62610933e-01,-2.51465380e-01,8.00231695e-01,1.77305222e-01,-1.08728492e+00,7.43145496e-02,-6.46809995e-01,2.45115548e-01,1.08661922e-02,-4.63938177e-01,-1.25569075e-01,6.59814954e-01,-6.11145377e-01,-3.65482531e-02,-6.81114495e-01,-2.29151353e-01,-2.92318285e-01,2.19541952e-01,2.04737306e-01,1.53066963e-01,-9.19904232e-01,2.90918529e-01,7.33642057e-02,1.34343468e-02,-6.75932989e-02,1.83801353e-02,-6.49115324e-01,4.95714128e-01,-7.04895496e-01,7.05112591e-02,-3.66950303e-01,5.00255764e-01,-6.28126144e-01,-3.10609877e-01,-7.12360144e-02,-8.41013640e-02,8.76546949e-02,-3.67774814e-02,-1.56631947e-01,-6.90774977e-01,1.95805147e-01,-6.15794025e-03,-1.15249741e+00,1.79952562e-01,-4.26430166e-01,-1.09286025e-01,1.13073148e-01,1.82587966e-01,-9.27230954e-01,-2.54984379e-01,-5.10816514e-01,5.44361211e-02,-5.93931794e-01,1.40334174e-01,1.97529644e-01,1.72172084e-01,-6.81511104e-01,2.96389103e-01,-6.83957636e-01,-2.84480780e-01,-2.69308329e-01,1.86663121e-01,-7.37842977e-01,6.57346785e-01,5.45735836e-01,3.52011979e-01,6.47770703e-01,2.49738634e-01,-5.85695982e-01,7.95342177e-02,3.63409556e-02,-3.58102262e-01,9.49751139e-02,3.33115608e-02,4.32528824e-01,2.04294696e-01,-1.05968758e-01,8.46447051e-01,-2.75742561e-01,5.37031770e-01,-6.33707285e-01,3.42337906e-01,8.20663154e-01,-7.96277404e-01,-4.44440469e-02,9.01198328e-01,1.70603305e-01,2.08388999e-01,8.11783969e-02,-1.17788061e-01,-3.72693062e-01,-4.52830911e-01],[6.92738146e-02,-1.21153474e-01,3.64757329e-01,3.34685057e-01,-6.10441089e-01,2.88220942e-02,-3.28829318e-01,-4.30354804e-01,-1.45946473e-01,-1.70181707e-01,-1.36633784e-01,-8.42158869e-02,-1.01420768e-02,-2.77315974e-01,5.95656484e-02,-5.04080392e-02,-4.65196222e-01,-2.30286345e-01,-3.00647974e-01,1.22175388e-01,-3.06861281e-01,2.41307616e-02,5.15505254e-01,-6.90916419e-01,3.49905074e-01,6.26542419e-02,-2.41617542e-02,3.83648396e-01,1.33381501e-01,4.13528591e-01,4.93676402e-02,-4.16742295e-01,-4.59804118e-01,4.18782060e-04,9.42849070e-02,-9.15608555e-03,7.60127246e-01,-3.42779398e-01,-3.40465695e-01,-9.77808610e-02,-1.23125343e-02,5.65990686e-01,5.12861550e-01,5.06303847e-01,1.08473264e-01,1.63885489e-01,-1.41056180e-02,-3.58881146e-01,-2.00391725e-01,6.35910332e-02,3.95160288e-01,-3.07161398e-02,7.37284064e-01,-4.52213109e-01,-4.06412661e-01,-6.11551255e-02,-1.50880888e-01,-3.40899855e-01,4.07931328e-01,-1.76913068e-01,3.78244907e-01,5.58175385e-01,4.28936541e-01,3.41600895e-01,2.58365333e-01,9.16806236e-02,-2.13372692e-01,2.06162393e-01,3.73183161e-01,8.46433938e-02,1.36962891e-01,3.47971380e-01,-8.21618438e-02,6.86026886e-02,5.29408395e-01,1.09198213e+00,-8.91273379e-01,-5.35311699e-01,-1.05743468e+00,4.49866831e-01,1.18345872e-01,-2.88558096e-01,1.33030429e-01,3.55740905e-01,-1.55099109e-01,-3.76352631e-02,-1.17141634e-01,-4.19307619e-01,-3.64509970e-01,4.02023494e-01,2.41185635e-01,-1.78182349e-01,-3.63167524e-01,-2.56731838e-01,-8.84904191e-02,-1.77943006e-01,2.97786742e-01,-2.65046537e-01,4.46076840e-01,1.64650857e-01,6.45940065e-01,7.02190161e-01,-3.34234327e-01,6.64970726e-02,4.33823705e-01,7.17485189e-01,-2.46619344e-01,-4.75364268e-01,1.45387143e-01,-3.20856899e-01,-1.91253230e-01,-2.31255040e-01,6.18859231e-02,-6.21411979e-01,-8.28802407e-01,-6.77889228e-01,2.89384156e-01,-8.28458190e-01,5.85495651e-01,-6.48460746e-01,-2.96527147e-01,4.45233494e-01,-1.91313848e-01,7.54198208e-02,-9.14710641e-01,-2.58716792e-01,2.68853843e-01,4.85350490e-02,1.20096616e-01,-1.06843755e-01,6.83740735e-01,2.66707599e-01,4.73012626e-01,-7.97752500e-01,5.14826238e-01,-7.87777066e-01,-1.54631004e-01,-5.34655094e-01,4.47783351e-01,2.84299791e-01,7.26272091e-02,-6.52855635e-01,3.69817726e-02,-3.09026539e-01,-9.94930342e-02,6.89752460e-01,-1.90444514e-01,-2.32304484e-01,3.90914857e-01,-6.51450515e-01,-1.88455313e-01,3.03571373e-01,-2.16655228e-02,5.77619374e-01,2.15355847e-02,2.76743352e-01,-1.26092181e-01,1.20260604e-01,-2.92731464e-01,4.29445505e-02,4.06896114e-01,-6.06275797e-01,-4.60715264e-01,-7.20347166e-01,6.61384583e-01,-4.17744100e-01,7.47200623e-02,1.18703611e-01,-8.11135530e-01,-3.56584132e-01,-2.00379774e-01,-1.79273963e-01,-5.80204539e-02,5.54932714e-01,-8.24379325e-02,-2.82557666e-01,5.27933314e-02,9.60826516e-01,-9.38119471e-01,-3.00241649e-01,-3.64998609e-01,-1.76717371e-01,-3.18067878e-01,3.61225367e-01,9.10716593e-01,1.48168206e-01,-8.05966854e-02,-3.33834320e-01,5.54948598e-02,-1.06741510e-01,6.83480799e-02,5.66541910e-01,-2.26862699e-01,-5.11780560e-01,-2.02503026e-01,-8.17921832e-02,7.34688789e-02,1.15853161e-01,1.93887483e-02,-1.01000190e+00,3.44809949e-01,-1.76204801e-01,-1.94880605e-01,2.72180885e-02,-4.48589206e-01,7.63768554e-02,5.76649681e-02,-7.83808053e-01,4.35128808e-02,-7.34459996e-01,-7.66733736e-02,-3.62083465e-01,-1.75075516e-01,1.90662622e-01,3.22682410e-01,-3.92027289e-01,5.48082888e-01,-8.35952759e-02,-4.12190296e-02,6.43014491e-01,-6.51332662e-02,-1.24083020e-01,-1.47298610e+00,-1.15143073e+00,3.73116702e-01,-1.02080524e-01,-1.42432630e-01,-1.07940102e+00,1.39875030e-02,-8.93909574e-01,4.97452796e-01,1.17996931e-02,-7.92799294e-02,2.82398641e-01,-1.25652909e-01,-8.27269256e-01,-7.88276866e-02,-1.41665488e-01,-3.81954998e-01,-2.29473948e-03],[1.0750071,0.35406548,0.35743776,0.1560095,0.13783225,-0.3243206,1.0177934,-0.77742064,-0.54929435,0.19050914,-0.09027344,-0.00702517,-0.07229466,-0.21348712,-0.34936282,0.19639592,-0.58704114,-0.3079937,0.27937052,0.19761407,0.25216532,-0.23267837,-0.2837918,-0.28868607,-0.1068025,0.25125778,-0.22292756,-0.3292118,-0.03783054,0.28614363,-0.08575066,0.26889542,-1.0746177,-0.04070633,0.44782537,0.3196088,0.2326552,-1.2256687,-0.01990747,-0.07608405,0.25229004,0.47862995,0.3280051,0.03879125,-0.6736175,0.03064822,0.23936851,0.3002827,0.64176273,0.21716891,-0.14404488,0.4505149,-0.3822488,-0.14308164,0.18065786,-0.34641898,-0.42730626,0.2801511,-0.1713419,0.11431497,0.28515738,-0.2286427,-0.52709615,0.15484807,-0.18053918,0.6725401,0.33661446,-0.06259178,-0.25936177,-0.06934072,-0.05776721,0.3045098,-0.146009,-0.3581802,0.3150854,0.17314957,0.28361285,0.05589609,0.00352861,-0.962548,-0.3364977,0.83043677,0.44145486,-0.2917071,-0.24284446,-0.49098697,0.13620171,-0.06845475,0.64236695,0.3972794,-0.51101834,0.1872944,-0.3398263,0.15031132,-0.08030984,0.15684281,-0.12969053,-1.3570073,0.22101867,0.01884345,-0.13854425,-0.5963744,-0.8731256,-0.5998128,-0.16093221,-0.79378563,0.28140163,0.08613396,0.39083228,0.08363103,-0.6258974,-0.06595891,0.07817928,-0.24081486,0.3255803,0.06778017,-0.6340033,-0.01023659,0.9041346,-0.47057995,-0.21169926,0.05224873,0.9348541,-0.42687845,0.04390253,-0.21614455,0.22565435,0.3769774,0.63615566,-0.43141904,-1.082863,0.11653013,0.34845507,0.35905686,-0.29657948,0.50946987,-0.45539173,-0.17706895,-0.4812496,-0.16806348,0.3693089,-0.22441855,0.0090125,0.43865433,-0.4641059,0.45057958,0.11514734,-0.5205284,-0.38921058,-0.12120015,-0.54573494,0.23594166,0.16649611,-0.39375675,0.4754449,0.16054514,-0.55299693,-0.26048762,0.06044317,-0.69720036,-0.01800039,0.24305928,-0.7624086,0.42884424,0.43399087,-0.4466888,0.04189911,-0.16143657,0.4858452,-0.2982278,0.38261402,-0.21353774,0.02518737,-0.73520154,0.37340197,-0.78945726,-0.5907229,-0.54437387,0.20770963,0.44203106,-0.34006652,0.3413249,-0.03533464,0.36840874,-0.29271758,-0.13659549,-0.12521979,-0.2197533,0.16555306,-0.01366369,-0.35414615,-0.09678384,0.5526378,-0.381445,0.31008875,0.07413438,-0.29199094,0.00475012,-0.3511122,-0.06562727,0.02010809,0.06970147,0.39320463,0.42303458,1.1219006,0.16444091,0.11110486,0.60863036,0.3279963,-0.985936,0.02050779,-0.36860284,-1.1096932,-0.50846004,-0.1619109,0.35614365,-0.84807956,1.0744034,0.29493695,0.28807095,0.14769597,-0.2623543,-0.23565705,-0.47875133,0.53992116,-0.42228347,-0.42628005,-0.13540794,0.03539548,0.04582619,-1.0079253,0.31809855,1.0257598,-0.6255452,0.09198041,0.14390908,-0.12343613,-0.25274456,-0.30926177,0.6324964],[0.11916942,-0.06518449,-1.2374883,-0.26113978,1.0980781,-0.30985746,0.57224035,-0.07345407,-0.30292046,-0.03348109,0.02573228,0.10290463,-0.24481311,0.24538407,0.12915364,-0.6295064,0.2213663,-0.52121437,-0.64539427,-0.31773466,0.00863819,-0.43824896,-0.8762451,0.31986248,-0.5489113,-0.30561918,0.46380922,0.15256473,-0.27631083,0.37739167,-0.06078693,0.21116118,-0.89908445,-1.0031255,0.3159177,-0.44776604,-0.05351722,-0.2526053,0.59139895,-0.23214914,-0.24857736,-0.49535233,0.21689197,-0.7004533,0.34129483,0.43236145,-0.1185448,0.3907458,0.37333095,0.34791806,-0.05178504,0.31436858,-0.356124,-0.08132329,-0.19159052,-0.09424372,-0.86447275,0.34731254,-0.32551298,0.7151843,0.19143996,0.15100932,-0.22260657,-0.02533228,-0.3542246,-0.69663894,-0.14270501,-0.20500027,0.06590749,-0.0658097,1.0497855,0.17427264,0.12435038,-0.01540984,-0.33246285,0.53719246,0.09181672,0.1891575,0.08966803,0.8016077,0.6254711,0.21646285,0.30699113,0.35007137,0.01393453,-0.32702988,-0.335626,0.33673286,-0.44588265,-0.4912747,-0.5112779,-0.87519866,-0.3645438,-0.3946799,0.08634382,-0.4402905,0.36895943,-0.32213497,-0.6158471,-0.20458491,0.6109688,-0.29721928,0.02803295,-0.80175453,-0.658237,0.7198114,0.29318264,0.9226531,-0.16191523,-0.33079112,-0.29312557,0.00889862,-0.50238127,0.21396057,0.19973075,0.4786588,-0.08985528,0.39736524,-0.16986331,-0.2976136,-0.7168638,-0.21510775,0.38480452,0.80745447,0.13269064,-0.45424172,-0.8012892,-0.1993165,-0.41645235,0.0590697,0.12287441,-0.01817382,0.20873974,-0.36499062,0.63485426,0.3133978,0.36497295,-0.44208664,-0.15489498,0.33092076,-0.68862355,-0.32151964,0.29167455,0.24171655,-0.51171523,0.2722488,-0.33275375,-0.20121476,0.03201198,-0.34352493,-0.12127325,-0.223995,0.16448283,-0.78925323,-0.5556226,-0.40767974,-0.5762149,-0.5197159,-0.15170568,0.7060057,0.3218272,0.13338323,-0.78934443,0.13768175,-0.38610092,0.20425723,0.48874846,-0.22924867,0.04275988,0.07860401,-0.1462776,0.7429057,-0.4476253,-0.07404043,0.06279431,0.30313846,0.26032627,0.46399903,-0.63916373,0.02714903,-0.40896145,-0.6733692,0.5099178,0.6041711,-0.05018499,0.27862832,0.13454595,-0.07840197,-0.18622325,0.46222216,0.09694505,-0.4288221,0.15381145,0.00283207,0.26873413,0.25765222,-0.29789066,-0.21983977,0.17290735,-0.57013595,-0.0435085,0.28607094,-0.30599415,-0.52207583,0.10291149,-0.55694383,-0.86720085,0.52222383,0.0937032,0.06101523,-0.38091812,0.34700483,0.07551033,0.41227254,-0.17255014,0.18826964,0.0951144,-0.07868452,0.2504651,-0.0512761,-0.33851004,-0.7113137,0.09467067,-0.2283508,0.18708701,0.5135424,0.6648626,-0.39093828,-0.5087912,-0.5231806,0.72901565,-0.91839015,-0.37593475,-0.6363071,-0.12950501,-0.35430354,-0.15558544,0.27005267,-0.44533864,-0.73631996],[-1.17125940e+00,2.05772102e-01,6.27880776e-03,3.68540525e-01,-1.08057936e-03,-2.40438491e-01,-5.94816059e-02,-7.68552721e-01,8.89490664e-01,-4.21645716e-02,3.05649370e-01,-1.19104540e+00,-1.39817074e-01,8.85005772e-01,3.13422047e-02,-2.38147736e-01,-5.65222986e-02,1.24438934e-01,3.85957599e-01,-3.01098138e-01,1.61320120e-01,-4.86638695e-01,-5.04178666e-02,1.53693065e-01,-1.38343856e-01,-7.67864883e-01,2.77683228e-01,-3.50765377e-01,2.91935578e-02,-3.06543671e-02,4.02537376e-01,-5.66555738e-01,2.27578148e-01,-1.16519965e-01,-6.78739101e-02,-1.59742385e-01,1.34489998e-01,5.70847690e-01,-5.47580235e-02,-1.85725644e-01,-8.13389383e-03,-2.53012925e-01,-6.01301014e-01,3.19382191e-01,8.39782059e-01,3.30214262e-01,4.47506607e-01,4.18989331e-01,-2.86622316e-01,-5.14124155e-01,-1.04461424e-01,-3.36566061e-01,2.19211012e-01,7.47148573e-01,-3.08395535e-01,3.46933275e-01,5.10205068e-02,-1.69089297e-03,-8.51660073e-02,-3.57265711e-01,-2.63778791e-02,-1.50818765e-01,-6.30082190e-01,-2.14614838e-01,-1.05593810e-02,1.07180500e+00,-2.72560775e-01,-1.17184401e-01,6.04945660e-01,4.26216602e-01,7.39694238e-02,-2.96246678e-01,3.22491020e-01,2.60280281e-01,2.82270342e-01,3.73786166e-02,2.03662127e-01,-2.93275654e-01,3.48515213e-02,3.16800326e-01,1.71939045e-01,4.58283901e-01,-9.31598485e-01,2.39865035e-01,3.36820364e-01,-9.75697339e-01,3.13870646e-02,-2.19257742e-01,7.41597831e-01,-1.13285625e+00,2.77189702e-01,1.54000521e-02,-5.13185263e-01,-1.86266437e-01,-1.92033976e-01,-8.16864520e-02,-1.09119751e-01,1.62925497e-02,4.90009606e-01,8.97769690e-01,-1.58005789e-01,4.07332748e-01,1.96861789e-01,1.22631863e-01,3.54537845e-01,-4.10629272e-01,1.20694213e-01,3.19972605e-01,-3.84548157e-02,1.59313038e-01,-6.24112077e-02,-2.73813941e-02,1.78551182e-01,-6.47238612e-01,1.54441983e-01,2.87881732e-01,6.63649943e-03,-4.54250038e-01,-4.26431358e-01,4.86654431e-01,3.82960849e-02,-9.77936164e-02,-1.41459942e-01,-2.31588647e-01,-6.98667049e-01,8.45943764e-02,6.39696941e-02,1.60537567e-03,-5.61425127e-02,-2.95428276e-01,-3.12056214e-01,5.25469184e-01,-4.63533461e-01,5.58522105e-01,-4.02450085e-01,3.53703886e-01,2.05630496e-01,-3.86297882e-01,-4.39409077e-01,-4.92964745e-01,2.06252143e-01,-4.64300364e-01,3.18934083e-01,2.66787022e-01,3.74990821e-01,-1.94580723e-02,-5.39198339e-01,4.62807231e-02,1.05299599e-01,-2.91341990e-01,5.26947796e-01,-1.65188104e-01,2.53688157e-01,4.51634020e-01,-3.03709265e-02,-2.30044901e-01,3.72639060e-01,3.60844225e-01,3.62748832e-01,-1.11172982e-01,1.45800963e-01,1.04022071e-01,2.52976477e-01,4.25350904e-01,3.13187271e-01,-1.54416449e-02,-3.54968041e-01,7.06232667e-01,-1.48999035e-01,-2.46972904e-01,4.36118603e-01,2.95802146e-01,1.50395678e-02,2.61720508e-01,-7.89683610e-02,3.05556774e-01,-7.71839172e-02,1.27393216e-01,-4.09656256e-01,7.37737358e-01,-3.11625689e-01,-2.12832242e-01,-3.98971401e-02,-1.10226050e-01,1.57039866e-01,-6.89542770e-01,-5.40852487e-01,-2.86796510e-01,-4.74070817e-01,6.09963715e-01,4.36837465e-01,7.36256018e-02,2.04145722e-02,-2.80644428e-02,1.76536664e-01,-4.40363288e-01,1.13777965e-01,2.53821433e-01,-1.14946052e-01,3.56473953e-01,1.83829620e-01,1.10317683e+00,4.47687835e-01,5.89437425e-01,-4.41324189e-02,-6.04207478e-02,-4.50398445e-01,-7.55347759e-02,-4.91859019e-02,-7.38906860e-01,4.40178305e-01,6.96250871e-02,2.89503813e-01,2.38837168e-01,-7.41301000e-01,-2.02189133e-01,4.03758734e-01,-7.96253681e-01,-5.53233027e-01,-1.25827268e-01,4.32110913e-02,-1.55798808e-01,9.55104530e-02,7.65008405e-02,1.50731772e-01,-8.60862255e-01,-2.66886562e-01,-1.33338034e-01,-4.63450253e-01,4.44631457e-01,4.30959791e-01,-1.06072474e+00,-1.79880351e-01,1.19218118e-01,-2.98374534e-01,6.04598641e-01,1.77196100e-01,-1.78504199e-01,4.21229273e-01,4.08370882e-01],[-1.18100309e+00,1.93903551e-01,-3.13290149e-01,1.81533843e-01,4.21292275e-01,-2.50590116e-01,-1.96467996e-01,7.04727948e-01,-4.23085183e-01,-3.66224110e-01,4.68845010e-01,2.45675206e-01,5.79700395e-02,1.11642039e+00,-3.73101115e-01,1.04284987e-01,-1.23001955e-01,4.54223484e-01,-7.29235530e-01,3.37142646e-02,3.02031904e-01,-7.12710738e-01,2.43151501e-01,2.42716208e-01,2.73051648e-03,-4.15353030e-01,4.65856194e-01,-4.08235073e-01,3.12338918e-01,-7.14369655e-01,1.38955057e-01,6.77908242e-01,3.83347452e-01,-2.73830354e-01,6.04160786e-01,8.29456490e-04,3.10514927e-01,3.74821685e-02,-5.65319598e-01,-2.14323010e-02,-6.26157880e-01,-2.67576844e-01,9.60509032e-02,-1.57951385e-01,2.96747297e-01,5.98651469e-01,1.42185718e-01,1.91921458e-01,-5.47548532e-01,2.20124513e-01,-3.44612151e-01,2.46029451e-01,-2.41675049e-01,-1.29297644e-01,1.19431578e-01,-2.42539421e-01,5.19750938e-02,-1.41930044e-01,2.08984405e-01,-2.93575436e-01,1.76913619e-01,5.96807837e-01,-3.66644263e-02,-1.96891993e-01,3.81795973e-01,-2.55078912e-01,-7.39851654e-01,6.13828063e-01,-4.97143865e-02,-7.81473219e-01,4.67634439e-01,-1.06063336e-01,5.67218065e-02,-5.05441725e-02,4.43778932e-01,-3.10512602e-01,-1.02211922e-01,-5.38050719e-02,4.78802741e-01,-1.13840170e-01,5.65016627e-01,-9.33463991e-01,-4.34218608e-02,-4.56075490e-01,-1.53973833e-01,-1.54980764e-01,4.03111190e-01,-3.85040551e-01,1.14969723e-01,-3.69607925e-01,6.69905543e-01,-1.87009618e-01,-1.00630678e-01,-4.66079228e-02,4.16900516e-01,3.32014561e-01,-2.20646501e-01,2.98763722e-01,-4.93269384e-01,-9.98400375e-02,5.17277598e-01,9.41836119e-01,-4.29692507e-01,-3.39020520e-01,1.15833022e-01,-3.33675563e-01,4.37849581e-01,-2.84112185e-01,4.28716868e-01,-7.98453391e-01,-1.22270979e-01,-3.48589897e-01,1.76372509e-02,-6.18927777e-01,-4.31969129e-02,-9.46743309e-01,-4.28732902e-01,5.04027367e-01,4.46098715e-01,2.75677145e-01,1.92910239e-01,-8.70951563e-02,-5.40365353e-02,6.39913857e-01,-7.91489519e-03,-2.28184044e-01,-1.56028897e-01,-5.34776211e-01,2.79066235e-01,-2.56206930e-01,2.47685332e-02,-9.69556905e-03,1.29700232e+00,1.10102952e-01,3.08722973e-01,1.18853159e-01,2.26724297e-01,4.64030743e-01,-6.10582232e-01,-1.50365487e-01,-1.37536481e-01,-8.24266553e-01,-6.73811913e-01,-3.29880148e-01,1.39507249e-01,3.72235030e-01,4.61505830e-01,-1.59833819e-01,2.22655937e-01,-7.27702260e-01,3.45520586e-01,1.42033696e-01,2.93205291e-01,4.21077311e-01,7.26157606e-01,-4.91537273e-01,-6.52095795e-01,-1.10236881e-03,1.28929511e-01,8.48959386e-02,2.27336109e-01,1.26665756e-01,2.76400864e-01,1.12049687e+00,-1.40075877e-01,-7.71949649e-01,8.52847397e-02,6.84630752e-01,-3.14722329e-01,2.15733916e-01,3.03017646e-01,-4.80998755e-01,-4.75657612e-01,7.62088224e-03,-1.46126330e+00,-3.60633075e-01,6.01190746e-01,-1.98366821e-01,-8.46876621e-01,-5.65997839e-01,-3.96376401e-01,-1.57064945e-02,-1.30870059e-01,3.12941641e-01,-3.11469346e-01,-1.22891866e-01,-3.87344748e-01,3.63558114e-01,-6.92950308e-01,6.12984657e-01,-2.93902397e-01,4.65971291e-01,1.40238017e-01,9.51486081e-02,8.24391246e-02,-2.42421702e-01,2.58393645e-01,-1.60095736e-01,3.71234626e-01,-2.67317314e-02,-4.85774606e-01,-4.38094199e-01,-3.02155644e-01,-1.59834936e-01,6.82958543e-01,-6.06444716e-01,7.68773258e-02,-4.17675436e-01,-4.08610076e-01,2.71798335e-02,-2.08816305e-01,-2.00620025e-01,-1.94706187e-01,2.98910558e-01,-1.17560661e+00,2.27384999e-01,1.15367122e-01,-1.03176212e+00,-1.60772249e-01,-5.36941707e-01,4.55434382e-01,-6.93259835e-02,2.48165771e-01,-3.79448503e-01,2.12603465e-01,-2.02920616e-01,-9.51592550e-02,1.58849642e-01,1.69145152e-01,-8.76617581e-02,4.68689855e-03,8.49789754e-02,7.08953500e-01,4.69702601e-01,2.54287794e-02,-7.28244901e-01,-3.60550702e-01,4.45648342e-01,-2.55000293e-01,4.57833886e-01],[4.25421089e-01,-2.22599149e-01,-7.95654714e-01,2.45103762e-01,5.68852648e-02,-1.13884270e-01,1.53158739e-01,-2.38211855e-01,-3.02135080e-01,-2.08609086e-03,1.63342003e-02,-4.85237628e-01,-1.41061947e-01,3.92082363e-01,-5.17823517e-01,2.37133384e-01,1.72468647e-02,-4.56225732e-03,-5.67106843e-01,-1.56179741e-01,-1.51628435e-01,2.82910407e-01,8.30354393e-02,8.05373117e-02,-3.45160484e-01,2.38832206e-01,4.44866940e-02,-2.59597242e-01,-6.70014918e-01,2.97530089e-02,-2.13904753e-02,-2.12682083e-01,-4.41916734e-01,6.68921173e-02,-4.65903997e-01,6.38337612e-01,7.29703844e-01,3.01125437e-01,-1.75386578e-01,2.20840439e-01,1.49746001e+00,-7.06218481e-01,4.69784290e-02,4.39129770e-02,-1.30004302e-01,-5.02812386e-01,-6.28188968e-01,-4.18738842e-01,1.13348770e+00,-3.80178899e-01,2.37078696e-01,5.69081247e-01,7.87741005e-01,-6.70334756e-01,-2.53321022e-01,1.62880540e-01,2.12236241e-01,-4.88157213e-01,-1.16737977e-01,3.84033829e-01,-1.64549023e-01,-2.37219751e-01,-4.96220887e-02,4.40772735e-02,-1.28497019e-01,-4.04988408e-01,1.96536735e-01,5.68741560e-01,-1.13655078e+00,6.58381462e-01,-1.00619100e-01,9.17302668e-01,-6.05286956e-01,-4.37304527e-01,2.93886036e-01,1.95555866e-01,2.30836123e-01,-2.35899501e-02,-9.43672806e-02,2.09583685e-01,6.20865047e-01,2.26812273e-01,6.48102313e-02,1.24988005e-01,2.75900334e-01,7.50441402e-02,-1.06486008e-01,8.90879780e-02,9.42227185e-01,-3.05962980e-01,1.48669243e-01,-2.54902244e-01,-9.64693427e-02,4.17995006e-02,1.66594520e-01,-1.25563189e-01,1.43019870e-01,-1.33042149e-02,-1.22953735e-01,-3.69436860e-01,-3.95704340e-03,2.38643765e-01,-1.49318159e-01,3.70618363e-04,1.21036887e-01,8.53156030e-01,-4.22783583e-01,2.78164089e-01,3.02940428e-01,3.16153288e-01,1.15563773e-01,1.03199768e+00,-8.53278190e-02,1.37309432e-01,-2.53179044e-01,-2.55237728e-01,-3.45494715e-04,1.81430221e-01,9.74903166e-01,7.78291374e-02,-2.75337517e-01,2.47487966e-02,6.82945549e-01,2.49310389e-01,-3.17336649e-01,4.49223787e-01,7.14129210e-02,-3.09845746e-01,5.73269976e-03,8.25737230e-03,-3.38008404e-01,1.74424559e-01,-4.99169260e-01,2.66481787e-01,-2.38507569e-01,1.72619671e-01,-1.14458777e-01,3.61390412e-01,3.27391922e-01,4.91168678e-01,3.72885354e-02,1.55849561e-01,-1.01706013e-02,-2.66888767e-01,-8.21743548e-01,-6.11609891e-02,4.92189080e-01,2.54805595e-01,-2.67131686e-01,-1.30833700e-01,-9.95151043e-01,4.95159090e-01,7.51167119e-01,-1.60443500e-01,3.46716046e-02,-3.68808769e-02,4.23889399e-01,2.70202965e-01,-7.76636228e-02,5.96183479e-01,1.86158553e-01,2.29151070e-01,4.36611287e-03,1.57308802e-01,-5.68444878e-02,2.49746084e-01,4.69411314e-01,-6.26781210e-02,5.12032449e-01,-5.31790018e-01,-3.96803468e-01,-2.48745829e-01,5.02228200e-01,1.42675653e-01,4.73918796e-01,-5.85503459e-01,-1.23017246e-03,1.46402791e-01,4.81335074e-01,-5.22936583e-01,3.18139084e-02,1.58235013e-01,-2.27712184e-01,-3.52997519e-02,2.93108404e-01,-2.59490937e-01,-4.83358949e-01,3.86912823e-02,1.14867508e+00,3.89287993e-02,2.41308212e-01,-7.27975219e-02,6.90646768e-01,6.18470050e-02,-1.63663268e-01,-3.24674636e-01,4.51911986e-01,-1.57743767e-01,2.82023400e-01,6.46899566e-02,1.68463007e-01,1.25838667e-01,-4.82280463e-01,3.86502266e-01,3.90408754e-01,-4.97913092e-01,2.79968858e-01,3.43614340e-01,2.26613477e-01,-7.76909292e-02,-1.53678671e-01,-2.40941629e-01,4.02272940e-01,6.16308331e-01,-2.70989776e-01,-4.63981666e-02,2.41041198e-01,3.33733529e-01,-4.54099923e-01,1.18439808e-01,7.40198940e-02,5.87996244e-02,-3.36933851e-01,-7.87935182e-02,2.87224144e-01,8.47702846e-02,1.02708049e-01,2.64700174e-01,7.77506113e-01,4.37439650e-01,-3.36180441e-02,1.36174306e-01,-3.43768634e-02,-7.24817991e-01,-4.68539819e-02,2.91440282e-02,6.38114065e-02,-7.89267778e-01,2.82543004e-01,-3.45030129e-01],[3.33316714e-01,8.86678576e-01,-8.05510581e-02,5.06748974e-01,8.82249773e-02,8.54431689e-02,8.79290253e-02,-1.73721135e-01,-6.05253279e-01,-1.09346673e-01,2.97477990e-01,3.02518755e-01,4.02286410e-01,2.64304876e-01,-1.05819893e+00,-1.43516839e-01,-3.53232265e-01,5.72946742e-02,6.91210181e-02,-1.53234929e-01,2.50936784e-02,1.29810914e-01,4.45124507e-01,-1.97583698e-02,1.62324026e-01,-1.63071886e-01,6.75710380e-01,-1.39906943e-01,1.76020965e-01,-1.34001104e-02,-4.69323933e-01,-9.97431949e-02,3.31690639e-01,1.21031620e-01,-3.24085116e-01,-7.38762975e-01,5.41156769e-01,-2.16380835e-01,-2.44131894e-03,-1.15736969e-01,-2.82814741e-01,5.01475215e-01,-2.18850017e-01,-2.61266291e-01,3.96242142e-01,-1.69327319e-01,-3.06351423e-01,-3.52212526e-02,-3.84289473e-01,-2.58710742e-01,-3.66544649e-02,4.20813292e-01,2.90973097e-01,-9.70379263e-02,-1.99692249e-02,-3.20465595e-01,1.75676674e-01,4.80505526e-01,3.41095537e-01,-4.98210222e-01,2.61190653e-01,3.38146627e-01,3.90060917e-02,2.08466426e-01,-3.67192805e-01,1.98859707e-01,-4.48568732e-01,2.47715712e-02,-1.25630647e-01,-1.42977551e-01,-9.86293033e-02,-1.16357528e-01,-6.26916170e-01,3.27233851e-01,-6.96293339e-02,-3.21872562e-01,1.58020630e-01,1.51359916e-01,-2.77379304e-01,3.78833055e-01,2.03838944e-01,-1.41361922e-01,7.89437532e-01,-2.41754398e-01,-3.85286272e-01,9.15697634e-01,-1.52569637e-01,-5.07577538e-01,6.60568893e-01,2.19854251e-01,-1.00148833e+00,-8.90828550e-01,1.76481307e-01,-6.81710303e-01,-4.48380393e-04,4.07491848e-02,-8.68087888e-01,2.01019365e-02,-6.38603389e-01,9.70054626e-01,-6.22824371e-01,3.46424550e-01,1.94409005e-02,-3.82060409e-01,2.10442320e-01,1.30370378e-01,-2.57372171e-01,3.93830061e-01,-2.39702091e-01,1.50709167e-01,-1.60569325e-01,-3.23233932e-01,-2.47481875e-02,1.73373055e-02,-1.55679271e-01,2.52848655e-01,6.16822839e-01,1.58204377e-01,-7.90461540e-01,-1.39336437e-01,4.10505891e-01,2.65317321e-01,1.49602681e-01,1.09782793e-01,-1.14138198e+00,-2.22256467e-01,-5.15622079e-01,-1.48809239e-01,2.83304960e-01,-3.12321693e-01,1.05608664e-02,3.52151752e-01,-3.39997768e-01,-9.69266985e-03,-4.92576361e-01,1.24201015e-01,4.47268993e-01,5.12393534e-01,-2.72562444e-01,5.89091837e-01,5.10065794e-01,4.05897170e-01,2.18579024e-01,1.39677241e-01,1.97202578e-01,-5.59503317e-01,1.71330065e-01,5.75510785e-02,5.92023969e-01,-3.59195709e-01,7.10272372e-01,3.49998362e-02,2.26209730e-01,-4.10335243e-01,-9.22538161e-01,-4.40277696e-01,4.91432786e-01,7.25929558e-01,-5.51922262e-01,-1.10697731e-01,4.33050811e-01,5.62790811e-01,-2.24358030e-02,-2.88388252e-01,6.18614294e-02,1.77519187e-01,5.23309827e-01,4.70721051e-02,4.37185280e-02,-6.40693158e-02,-3.17240387e-01,-1.50029317e-01,-1.94072276e-02,-1.02485228e+00,5.08984067e-02,7.54458755e-02,1.11147976e-02,-4.87940192e-01,-2.38038957e-01,1.22104660e-01,-1.49526400e-02,9.98389721e-02,3.51432353e-01,-1.66406050e-01,-3.02247703e-01,6.31341875e-01,-1.83607176e-01,-3.26842576e-01,-1.72616732e+00,5.71215749e-01,-3.03513139e-01,-6.84351206e-01,1.02146995e+00,-4.23110604e-01,5.07069886e-01,-1.61057696e-01,-5.29213309e-01,6.98477849e-02,1.43800601e-01,-6.24629438e-01,-1.36195469e+00,-5.13124943e-01,-3.97203982e-01,-1.52227841e-02,-6.39306158e-02,3.73863429e-02,1.40791014e-01,-3.29586118e-01,4.62752432e-01,-2.36444771e-01,-6.14614069e-01,-6.20411098e-01,4.77162004e-01,1.57778949e-01,-3.85971546e-01,2.01658458e-01,6.36335969e-01,1.24085676e-02,-4.64277238e-01,6.28562048e-02,-2.47138180e-02,-3.82832319e-01,1.12304099e-01,-4.66529950e-02,5.08344211e-02,-5.27906299e-01,-4.26856279e-01,-1.94572777e-01,-1.86677426e-01,-7.83112645e-01,4.54277724e-01,-7.90610075e-01,-3.73595860e-03,-3.66945654e-01,-3.35750759e-01,2.45644242e-01,-4.03658181e-01,-3.13083887e-01,1.48258001e-01,-3.29292506e-01],[-1.9797865e-01,-4.0452871e-01,-7.1603328e-01,-1.6225760e-01,3.0585238e-01,-2.7575913e-01,-8.3860511e-01,4.8456502e-01,-1.6381128e-01,-1.1474630e+00,-9.3957740e-01,7.1960032e-02,9.9048577e-02,-4.7146893e-01,2.7864701e-01,8.5302252e-01,4.4552496e-01,-8.3611682e-02,-2.4033259e-01,-1.6241859e-01,1.6446689e-01,-1.6942935e-01,5.8784246e-01,6.1368998e-02,-1.7650370e-01,-1.8725115e-01,2.2155927e-01,-2.4212463e-01,-6.1276287e-01,-1.0425807e-04,-2.0317174e-01,3.0079353e-01,-1.5729445e-01,3.3383101e-01,-5.8646870e-01,-6.2827015e-01,-4.9935877e-01,-2.5460890e-01,8.5758135e-02,-6.6614032e-02,-4.1390893e-01,-2.8079867e-01,4.9886787e-01,-2.7883399e-01,-8.4974784e-01,-2.4050945e-01,3.6460462e-01,-7.2467607e-01,7.1822390e-02,-2.7331099e-01,7.7916110e-01,-7.3866749e-01,-5.7031494e-01,1.9540203e-01,6.5174687e-01,-2.0205948e-01,-7.4319345e-01,2.4975650e-02,-4.2852983e-01,7.5246656e-01,1.8446445e-01,1.5687616e-01,-2.2569823e-01,5.6782109e-01,-4.5379144e-01,1.3696657e-01,-1.3125229e-01,7.6916295e-01,-8.7613210e-02,4.1124779e-01,5.6430084e-01,-1.1157723e-01,4.8307562e-01,-8.4180194e-01,4.5456547e-01,8.0696031e-02,5.7978541e-01,-1.8908073e-01,4.2757717e-01,-1.1478441e-01,-1.8449669e-01,-2.2329910e-01,-1.8614018e-02,-2.6084149e-01,1.5497711e-01,6.5245531e-02,2.8296661e-01,-6.0248232e-01,-2.9220188e-01,-1.4194955e-01,4.2663774e-01,1.5807419e-01,-1.3092892e-01,-3.0083659e-01,-2.3556718e-01,-1.5788198e-01,-3.2836056e-01,-4.0901384e-01,-3.2708147e-01,9.1632342e-01,4.3481568e-01,1.8564224e-01,4.6184278e-01,-4.4686976e-01,-9.1551185e-02,1.7346501e-02,2.1116218e-01,-7.3548399e-02,3.1757289e-01,-1.5055047e+00,-2.2718513e-01,-2.4347191e-01,-3.6767864e-01,-4.2629609e-01,-3.4894991e-01,-1.4129803e-01,9.0115207e-01,7.7562356e-01,-2.3391883e-01,1.9386029e-01,2.2019361e-01,5.2012634e-01,-8.0263764e-01,-7.1856666e-01,1.7529190e-01,-8.2906723e-01,-5.4080778e-01,-6.7281115e-01,-6.0108203e-01,4.8960504e-01,3.2170035e-02,1.2684450e-01,-3.8002267e-01,-8.7222137e-02,6.1211906e-02,2.7701119e-02,3.9685902e-01,-1.0224567e-01,-2.6081192e-01,3.5604492e-01,-1.1462017e-01,-8.3479774e-01,4.2148780e-02,5.1366460e-01,-5.8240615e-02,1.3449056e-01,-4.3867746e-01,2.7787948e-01,-5.9907448e-02,8.0277658e-01,-8.7070483e-01,5.7248348e-01,6.2158391e-02,2.7887481e-01,-8.8395588e-02,-1.5978384e-01,1.0854065e-01,5.9104770e-01,1.2056751e-01,4.9822330e-01,2.3275065e-01,1.3840064e-01,4.9615443e-01,-4.8040712e-01,-3.2429016e-01,2.1995072e-01,2.2165336e-02,-2.7572623e-01,-2.9641455e-01,-1.1257218e-01,-4.9309626e-01,1.6083868e-01,-2.7029011e-01,3.9904216e-01,6.0685791e-02,3.7918094e-01,-8.7825274e-01,-5.0187808e-01,1.1457153e-01,-4.4344131e-02,1.0291488e-01,-3.6545080e-01,-4.2407504e-01,1.4997125e-01,-9.9062157e-01,-2.2634226e-01,2.0681486e-01,4.3218318e-01,3.1147996e-01,-1.0570595e+00,3.9397776e-01,4.0310256e-02,3.8811496e-01,3.6050665e-01,1.6737480e-01,1.9013846e-01,-1.0645902e-01,5.3816926e-01,3.0971131e-01,-4.0478981e-01,-3.5429963e-01,9.8751701e-02,3.0366439e-01,-3.4369761e-01,2.5835750e-01,2.5679129e-01,2.9361674e-01,-1.4491016e-01,-3.6880019e-01,-5.0858271e-01,2.8548444e-02,-8.3601648e-01,-5.7493848e-01,6.3810563e-01,5.9846854e-01,2.1421994e-01,-1.3248527e-01,9.2334472e-02,-2.5784782e-01,-4.4188616e-01,3.4767467e-01,7.5747120e-01,-6.9977647e-01,-6.8239354e-02,1.4341061e-01,1.1380369e-01,1.5534608e-01,3.6897907e-01,9.1682673e-01,2.1761385e-01,-6.8515861e-01,-9.8251373e-02,-1.6873549e-01,2.2410679e-01,3.4940594e-01,4.1426840e-01,-2.5695806e-02,2.7524358e-01,3.6669648e-01,1.2671813e-02],[-0.20502912,0.25092068,-0.58787316,0.05540587,-0.86454034,1.1317685,0.06446176,0.3604269,0.19561036,-0.3232303,0.02768187,0.69549674,-0.24021558,-0.04667543,-0.15394224,0.16205081,-0.93184155,-1.1028439,0.5643879,-0.10810092,-0.3391549,0.3661234,0.06460305,0.31041634,-0.77783245,0.12367824,-0.6881124,-0.11081014,0.53719336,0.13826263,-0.1849802,-0.40070426,0.40109637,-0.41309372,0.14378294,0.17486311,-0.68922806,-0.4176601,0.46094337,0.2625641,-0.03473687,-0.32946748,0.45094532,0.27596137,-0.13396443,-0.01719661,0.06045074,0.8179541,-0.31090423,-0.25982,-0.06474094,-0.6203588,0.03771649,-0.19508626,0.23758103,0.18786447,-0.8086795,-0.01223589,0.14928272,-0.15980893,0.3764629,0.04033136,0.01457261,0.18154418,-0.7598464,-0.02284824,-0.8163738,0.3583958,0.0174071,-0.01748398,0.26076648,0.14384131,0.28473163,-0.7031793,0.20494294,-0.59673446,0.40972677,0.15730776,0.19534808,0.35710183,-0.27481157,0.16847925,-0.05308104,0.26325363,0.02683584,0.05391496,0.16090454,0.20369738,-0.04673325,-0.10252663,0.03955295,-0.09803107,-0.13044415,0.30778,0.2021514,-0.67585355,0.2830325,-0.1738206,0.0821848,-0.04526835,0.5940882,-0.69220906,-0.1215848,-0.5020796,0.29947212,0.37308607,0.13945174,0.44184354,-0.43907806,0.9021521,-0.9058716,0.25802293,-0.3198371,-0.5539288,0.11104792,-0.55990374,0.250654,0.00777777,-0.95103115,0.53831315,0.2122476,-0.21676613,-0.76753676,0.10390228,-0.23163766,0.26123682,-0.47807226,0.08170897,-0.2973481,0.60318553,-0.11812697,0.2185294,-0.18182775,-0.09719803,0.0038885,0.36562073,0.10117161,-0.2610447,-0.17104645,0.14019568,0.40542457,-0.33360216,0.540099,-0.296242,0.47924158,0.7273764,0.01570053,-0.5843658,0.2827765,-0.29264846,-0.27776867,-0.7269078,-0.39962283,0.42816004,-0.5299779,0.02700598,-0.64648247,-0.04271601,0.80162853,-0.05434967,-0.28086495,0.22083175,-0.5389757,0.74838316,0.46033546,0.11965176,-0.1891164,0.08832868,0.03652128,0.26303005,0.18703434,-0.7289341,-0.20907569,-0.05707512,0.15720999,0.5038236,0.38753325,0.34029877,-0.10661508,-0.6479981,-0.30566034,-0.5647113,0.28672343,-0.04546713,0.13142288,0.35285917,-0.0961921,-0.02260158,-0.2774123,-0.67942876,0.18404989,0.488283,0.01525501,0.48722467,0.03293679,-0.02588509,-0.4661532,-0.32475033,-0.4355311,0.668404,0.3922411,-0.595528,0.11392206,-0.57242304,0.1044132,0.31834954,-0.39720464,-0.13543066,0.54531115,0.31599447,-0.3153745,-0.04264797,-0.0869361,-0.7824728,0.0960234,0.41114104,-0.513409,-0.08917655,-0.5455762,0.05400474,-0.57741904,0.18223628,0.09034643,-0.22029246,-0.00345451,0.707588,0.11639953,0.21907946,0.36802796,0.2533702,-0.4696984,0.58593315,-0.2664992,0.46259135,-0.23049463,0.06940579,-0.10966009,0.58487123,-0.08788383,-0.66001695],[0.28586555,0.36338246,0.18332511,-0.13808322,0.2499088,-0.26957437,0.4057795,0.02095732,-0.04703724,0.5956305,0.41039222,0.48689824,-0.39394414,0.22566405,-0.6650005,0.14218424,0.6627704,0.3679712,-0.4494875,0.45661417,-0.41360492,-0.05905367,-0.03938009,-0.07373913,0.5186323,-0.65794975,-1.2404839,0.45322528,-0.5353393,0.554087,-0.19384404,-0.40086126,-0.45629945,-0.3530358,-0.00146583,0.5719085,-0.0190363,-0.03657764,-0.03226751,0.44646776,0.4378779,-0.14788133,0.19329704,-0.21249735,-0.47144383,0.50244373,-0.4259137,0.28056383,-0.6783996,0.10191724,-0.32696202,-0.23803575,0.3046635,-0.1810395,0.30188748,-0.4539013,0.05504203,-0.33279055,0.09019378,0.0045587,-0.10442644,-0.10232792,-0.14473072,0.3309224,-0.36638394,-0.521921,-0.14209354,-0.15849498,-0.2538799,0.01959484,-0.4486551,-0.784722,0.07699197,0.39743793,-0.4934,-0.82444406,-0.18636759,-0.22397141,0.02798197,-0.30885,-0.54688305,-0.5153409,-0.09666822,0.05097464,0.14409919,0.06176734,-0.03228111,0.01423084,-0.34974015,-0.58008343,-0.5578648,0.24416508,0.11286072,0.04057983,0.07670938,0.18494551,-0.14577383,0.2831598,-0.55764234,0.15435824,0.4631869,-0.11203589,-0.9290361,0.33504367,0.0912974,0.16600406,-0.11906455,-0.04847385,-0.6247656,0.18258809,-0.28849828,0.20877847,0.36880004,-0.04259665,0.38592362,0.04555518,0.11800413,0.70538914,0.16814059,-0.05555625,0.1255861,0.03389043,-0.5297573,-0.0030246,0.7528335,-0.0012547,-1.0874377,0.7749438,0.08790977,-0.25623006,-0.11283104,0.28004912,0.15846415,-0.36845568,-0.17521513,-0.26741517,-0.38864106,-0.34913993,-0.5102855,0.49943748,-0.16491155,0.10905593,0.49785355,0.11880152,0.04212716,-0.55159813,-0.13615108,0.09436139,0.39151803,0.1381596,0.24195796,0.5528496,-0.05876757,0.21304974,0.1280658,-0.20257273,0.23168492,-0.5088805,0.28915614,0.02543821,-0.60441506,-0.8551796,-0.17792623,0.09115738,0.41383612,-0.3744183,0.08518615,-0.34271064,0.10981024,-0.5809775,0.06653185,0.74672526,-0.5803891,0.91651475,0.5369857,0.10345765,0.8587579,0.22883646,0.45587242,-0.1324617,-0.8197951,0.40159357,-0.8228247,-0.27604938,0.37220734,0.28986958,0.0280928,-0.7647182,-0.37205753,0.34602684,-0.20562752,0.31110898,0.00216213,-0.31474996,-0.09144432,-1.0028459,-0.05825401,0.11868028,-0.6121473,0.28208724,0.7933245,0.10768531,0.67114794,-0.59784055,0.1997158,-0.24512027,-0.22503479,-0.14491987,0.30773836,-0.41072735,0.01979579,-0.3378781,0.08914867,-0.6248886,0.5272457,-0.22176649,-0.4299955,-0.44348416,0.04100796,0.08760104,0.5838692,-0.27271113,-0.7521771,-0.04267493,0.04965322,0.56152505,0.37514403,-0.500076,-0.2984383,-0.75281775,-0.10398415,-0.45970032,0.19717066,0.6206549,0.03543881,0.2388627,0.1438358,0.48512393,-0.31844297,-0.3562665],[7.59046932e-04,-3.90680760e-01,-5.97959310e-02,-8.00584137e-01,8.85499790e-02,-1.02222681e+00,8.49864110e-02,-8.35537314e-02,5.99052668e-01,-8.85878682e-01,4.25207704e-01,-3.71009558e-01,2.53433496e-01,-7.41179645e-01,1.21293301e-02,-3.73858601e-01,-5.73727377e-02,-1.71421632e-01,1.38569936e-01,2.90908575e-01,-3.35125893e-01,-6.37458265e-01,1.96049333e-01,4.47281778e-01,-5.63258603e-02,4.02065247e-01,6.96830973e-02,-4.12770599e-01,8.67467523e-02,-7.74842978e-01,-6.46140873e-01,4.11238134e-01,5.85111439e-01,4.22569305e-01,-2.00932294e-01,-5.11057317e-01,4.03387606e-01,5.64918458e-01,-5.63472748e-01,5.93583211e-02,6.07056692e-02,9.51339677e-02,-4.54488903e-01,8.85929689e-02,2.13587493e-01,1.29185468e-01,5.56394815e-01,2.39379257e-01,1.47995442e-01,6.22913361e-01,-3.02413076e-01,5.63777983e-01,-4.05195862e-01,1.43413618e-01,1.23759704e-02,1.60524379e-02,-4.82405037e-01,-6.74277902e-01,3.89923006e-01,-2.98813552e-01,4.96737361e-01,2.22149998e-01,-4.59771268e-02,1.97195597e-02,3.55572790e-01,-6.53536022e-01,-3.24429154e-01,6.20444119e-01,5.55791676e-01,-9.70721781e-01,-3.66208971e-01,-5.29478550e-01,-6.25142097e-01,-1.71871826e-01,4.12389338e-01,4.23153266e-02,3.75852674e-01,-2.48755082e-01,2.70202279e-01,-3.33120048e-01,-4.92211252e-01,-4.48907763e-01,-5.45268320e-02,1.32405147e-01,-1.07784569e+00,1.75739691e-01,8.69730394e-03,4.90906052e-02,-3.63541335e-01,2.47443154e-01,-1.24213383e-01,-1.52984247e-01,2.69981712e-01,2.67032325e-01,-9.02485996e-02,-2.53818274e-01,-1.49428725e-01,6.20111883e-01,2.71363020e-01,2.72438765e-01,3.87847692e-01,-1.02643400e-01,7.09032059e-01,4.25298154e-01,3.52687627e-01,-1.69327688e+00,-6.88779294e-01,2.06447825e-01,-5.72892986e-02,5.35915136e-01,1.68979824e-01,7.63810396e-01,-3.49780232e-01,-5.34097135e-01,9.74571481e-02,-1.92027371e-02,3.11117414e-02,6.02569103e-01,-6.30647242e-01,-4.64466661e-01,-1.91055298e-01,-7.34430730e-01,1.05836689e-01,5.68380177e-01,-8.59293461e-01,1.97406635e-01,4.36514407e-01,2.29173407e-01,1.09348215e-01,-2.99557537e-01,3.11381161e-01,-1.30793065e-01,6.82208389e-02,-4.21096563e-01,-1.35724217e-01,-3.61922234e-01,-9.74602878e-01,1.12035632e-01,-1.98837876e-01,-3.51565093e-01,-3.27863842e-01,-1.51867777e-01,-3.72392774e-01,-6.05929136e-01,3.18641454e-01,3.70991200e-01,4.70651649e-02,-3.84139150e-01,1.38455009e+00,-2.98533320e-01,3.74277771e-01,6.37769341e-01,3.18066701e-02,4.78940606e-01,-1.15133850e-02,-2.17884094e-01,-7.36961812e-02,1.40738741e-01,-5.21793067e-01,4.54454154e-01,3.88546586e-01,1.54333911e-03,1.00985929e-01,2.16341421e-01,-3.53853106e-01,5.65687478e-01,4.59248692e-01,2.76237845e-01,-1.82016969e-01,5.70586085e-01,-1.30950794e-01,6.67596757e-02,6.63224280e-01,1.29323630e-02,-3.44171464e-01,-3.18491459e-02,-4.72727001e-01,1.60761401e-01,-2.72256374e-01,1.01676233e-01,-5.53485304e-02,8.04157741e-03,1.30487606e-01,-3.39456528e-01,2.68580794e-01,3.50994527e-01,1.15714192e-01,2.02829801e-02,8.68441761e-02,2.50745267e-01,1.13617495e-01,4.62079281e-03,4.12409306e-01,-8.29126090e-02,-9.25121725e-01,7.06902564e-01,7.90740490e-01,3.43198627e-02,-2.09572930e-02,-5.78839958e-01,1.70753255e-01,-2.09529594e-01,-9.84416530e-02,-1.56353474e-01,-1.13826290e-01,5.58645666e-01,5.47620416e-01,8.37827474e-02,8.19196552e-03,-4.13614772e-02,1.82957873e-01,-3.63477528e-01,-4.54707563e-01,1.42072260e-01,3.72094154e-01,4.65174288e-01,1.00485396e+00,-4.40951526e-01,-2.81381041e-01,2.64363717e-02,-9.30729061e-02,8.30475315e-02,-4.47231919e-01,1.68812811e-01,2.05317691e-01,-6.81618035e-01,-5.11572778e-01,-1.41370118e-01,-2.96409100e-01,-1.55621767e-01,-5.94120383e-01,-5.73614120e-01,4.35935743e-02,2.01280657e-02,-1.19012222e-02,1.83945522e-01,1.22570820e-01,6.63488746e-01,1.31960526e-01,-6.39805570e-02],[0.01571321,-0.62264264,0.4306793,-0.43110663,-0.15721682,-0.79967576,-0.47465187,-0.28307706,-0.19998269,0.19367044,-1.0146223,0.3523256,-0.13491061,-0.07965767,-0.47536084,0.2505306,0.74350935,-0.39041635,-0.12800112,-0.0436008,-0.10274341,-0.10565557,0.00216797,0.18443343,-0.40714794,0.13969423,-0.3587762,0.6539059,-0.85862637,-0.26100403,0.4616865,0.06125942,0.1548366,0.4823786,-0.21577574,0.12623987,-1.0389383,-0.09074561,-0.09208333,-0.3401858,-0.31318897,0.01431917,-0.17932153,0.35974053,-0.21574348,-0.00729846,0.6982035,0.41148975,-0.3132661,0.2430007,0.2999855,0.00534493,0.07789207,-0.02296633,0.2636587,-0.5592619,-0.03041527,-0.46996054,-0.09299625,-0.62846595,-0.12263502,0.127845,-0.87980515,0.30057052,-0.33408695,0.18607566,0.47718245,1.2791919,0.86251456,-0.00232504,0.33708638,-0.03216006,0.7458968,-1.1634742,0.18371077,0.8582528,-0.36254248,0.2917243,0.2227285,-0.62329453,-0.10919996,-0.22854094,0.0549144,-0.04743677,-0.27715552,0.05366598,0.02674048,0.18737352,-0.44549707,-0.05740907,-0.5113063,0.23595388,-0.11018094,0.32648194,0.68894273,-0.43369758,-0.503849,-0.36430547,0.16486767,-0.68009967,0.30758828,0.16577181,-0.4661269,0.22914852,-0.51443326,0.2910546,-0.02215466,-0.27545157,-0.01068441,-0.21398923,-0.8466137,-0.38460043,-0.36690652,-0.52796644,0.05799356,-0.5386103,0.15287797,0.34643653,-0.5639753,-0.05144626,0.15679693,-0.06684394,-0.4000281,0.21616998,0.3086963,-0.7464093,-0.39742857,-0.28643066,0.15136561,0.6765119,-0.02413582,0.6085327,0.41098964,-0.25051156,-0.11880881,-0.17175744,0.14896227,-0.5650176,0.25266743,0.21835025,0.05723044,0.37424502,-0.3899078,-0.13976398,0.6801615,0.05667242,0.07659398,-0.81953734,0.3010701,0.26865765,-0.08044336,-0.31209445,-0.69022703,-0.26248568,-0.12362699,-0.0328965,0.16890973,0.07926983,0.33703086,-0.4060285,0.5946936,-0.15165095,0.04521886,0.6468801,-0.21762328,-0.48823175,0.13565381,-0.09532195,-0.10096452,0.2863776,-0.15764487,-0.5870897,-0.29422,0.5985617,0.6116284,0.43611223,0.30965805,-0.27700427,-0.66468245,-0.22797312,-0.50760484,-0.01919781,0.19750386,0.37676838,0.20496844,0.3249735,-0.06184124,0.5668359,0.07368866,-0.40125972,-0.1502049,-0.04169219,0.5757395,0.19570911,0.54366237,-0.07546659,-0.19696864,-0.36701366,0.04044264,0.46043536,0.20324798,0.36136615,0.23509188,-1.1252269,0.3925997,-0.58492017,-0.10468438,0.04608534,-0.08434842,0.30095527,0.8436222,0.1746743,0.25031045,-0.51808655,-0.56828797,-0.18970025,-0.17705302,0.18656443,0.07027033,-0.45124784,0.15805829,0.5319392,-0.287672,0.49951458,-1.7603853,-0.27516708,0.00519542,-0.39895627,0.32449278,-0.5971733,0.2624996,0.28286776,-0.03487186,-0.30263177,-0.05931117,0.3696671,0.35201392,-0.10090064,0.24719402,0.16463776],[-5.04738092e-01,-4.60855335e-01,4.48050946e-01,-7.33575821e-01,6.92752823e-02,2.81802356e-01,-1.21192682e+00,4.98216935e-02,5.32351017e-01,3.32482606e-01,3.11469942e-01,-4.17106897e-01,5.39647579e-01,-9.19893980e-02,3.86868119e-02,-3.46984982e-01,2.91033268e-01,-5.60472369e-01,2.27534205e-01,-2.56807327e-01,-6.05501942e-02,5.49294770e-01,-3.69434118e-01,-2.96911150e-01,-3.11660618e-01,-3.84813905e-01,2.89241195e-01,8.14082995e-02,5.00589371e-01,3.22059065e-01,-5.06453402e-02,-5.37433028e-01,3.31964880e-01,3.59101921e-01,-2.71357596e-01,-2.26251304e-01,-2.58321315e-01,7.87492245e-02,2.90290326e-01,4.80804235e-01,2.34990001e-01,-1.91894814e-01,-5.78949511e-01,-2.52396375e-01,4.78591204e-01,1.12742949e+00,-2.82335579e-01,-1.56470135e-01,1.32421013e-02,-3.84334803e-01,2.10203841e-01,-9.15742815e-02,5.32734871e-01,-1.90533012e-01,1.59310937e-01,8.81648302e-01,-3.75736207e-01,-4.64145169e-02,-4.73821253e-01,3.57297719e-01,-2.95543313e-01,3.48323807e-02,-1.29347602e-02,5.30410290e-01,-3.94783974e-01,-3.28107476e-01,-8.77025202e-02,-2.46950522e-01,-1.62044421e-01,-5.35771132e-01,-1.29320487e-01,8.22533146e-02,-9.35426712e-01,1.29398569e-01,-3.89873475e-01,-8.88343062e-03,1.33155078e-01,1.54591814e-01,-5.02462447e-01,-3.55471164e-01,-3.34605545e-01,6.00210316e-02,-5.04486918e-01,2.56950289e-01,4.18816626e-01,1.61674604e-01,3.63986999e-01,-4.33157712e-01,8.66224691e-02,-2.83725500e-01,5.55839427e-02,-1.21399529e-01,-5.39365597e-02,-5.09853289e-02,4.20124412e-01,-5.69061160e-01,5.90399444e-01,-6.48853481e-01,2.54500518e-03,-2.70573169e-01,-2.42773265e-01,1.43469527e-01,6.01332448e-02,1.80318672e-02,-6.91238120e-02,-5.01962543e-01,-3.62720996e-01,-6.34201884e-01,5.31393707e-01,7.66057789e-01,1.47458091e-01,1.57147393e-01,6.00064516e-01,-6.92675948e-01,1.54826909e-01,6.78033888e-01,-2.53143489e-01,-1.42446846e-01,2.28281152e-02,4.60046202e-01,2.03574792e-01,3.51265043e-01,5.62557042e-01,4.38413799e-01,-1.23780169e-01,-1.58130050e-01,6.93515614e-02,-5.11518121e-02,4.88454640e-01,9.46706384e-02,-6.24856174e-01,4.53719676e-01,-1.02875993e-01,-7.54785299e-01,-2.11914197e-01,-4.13298815e-01,-6.11861050e-01,-3.88991594e-01,2.53947586e-01,-4.84131396e-01,-2.86616087e-01,-1.19388235e+00,-1.54556796e-01,4.27265406e-01,2.99513221e-01,-3.29357177e-01,5.13546646e-01,-3.00935861e-02,3.84410560e-01,4.74802181e-02,-1.33114979e-01,-2.66563892e-01,-2.25545809e-01,-2.31459644e-02,4.01341885e-01,-2.84041256e-01,2.11890399e-01,-5.24572670e-01,-2.03099489e-01,-4.48491238e-02,2.41477519e-01,2.15277925e-01,3.68048877e-01,1.42227902e-04,-1.89490587e-01,1.22462995e-01,3.09664197e-02,-1.21670932e-01,-6.75131559e-01,-1.65479809e-01,5.04650414e-01,2.16379628e-01,6.85685337e-01,-3.44956927e-02,-1.12142749e-01,-5.94099820e-01,-9.90260243e-02,-2.20442131e-01,-1.49871811e-01,7.04639629e-02,-6.02567904e-02,2.83952922e-01,1.35614453e-02,2.33371675e-01,2.48339891e-01,-1.09073699e-01,4.55871731e-01,2.95292497e-01,-1.11761189e+00,-4.94060069e-02,-6.87892377e-01,-1.01567471e+00,-1.00938510e-02,-1.25761718e-01,-2.50826895e-01,1.78260744e-01,-4.36852276e-01,-3.53888720e-01,7.75842488e-01,5.46471700e-02,-8.45126629e-01,3.80424969e-02,3.27464044e-01,-3.08256954e-01,5.73914111e-01,2.25013912e-01,-3.38505417e-01,-6.34824336e-01,-4.57977206e-01,-4.63839501e-01,-2.06704214e-02,2.76751220e-01,1.70064829e-02,3.26718748e-01,-2.93968786e-02,-2.51362830e-01,-3.07710450e-02,7.63115346e-01,-7.02612847e-02,-2.35338956e-01,2.45247576e-02,-1.47505909e-01,-4.87089932e-01,3.48792374e-01,-7.22486436e-01,5.07282913e-01,2.38113955e-01,-4.88988668e-01,-5.77904284e-01,7.54665196e-01,-2.21278176e-01,-3.39918584e-01,-4.83851880e-01,4.41308245e-02,2.41370082e-01,7.08099127e-01,1.15382388e-01,2.01957330e-01,-5.62053740e-01,-7.16285184e-02],[0.11222047,0.5804261,-0.7117369,0.3464561,0.13316296,-0.04081039,0.07684572,-0.9269175,0.34281442,-0.49734655,-0.05420132,0.40768448,0.6345251,-0.24515024,-0.04127723,0.23755789,0.02522439,0.12768959,-0.31055588,-0.39468613,0.26762763,-1.3629655,0.46554756,-0.45173508,-0.14548926,0.71724844,-0.33055466,0.12512255,0.34884104,-0.42346227,0.1281593,-0.1300612,-0.6110049,0.35892406,0.42772666,0.44988298,-0.05575776,-0.24385609,-0.92278,0.33141252,-0.44363248,0.14901285,0.01442808,-0.8191599,-0.02023646,0.2840251,0.39004454,-0.23446047,-0.1272626,-0.14968729,-0.35122892,-0.4198046,-0.01483984,-0.48773074,-0.31011122,-0.37464687,-0.11420951,0.30619425,0.54541695,0.03068756,0.251939,-0.9056646,0.6853931,0.32362026,-0.14783107,-0.41688833,-0.24625844,0.7025755,-0.1323459,-0.82709134,-0.4325673,0.22028044,-0.60276043,-0.2993197,-0.02247043,-0.18118986,-0.3601614,-0.41098884,0.08880372,-1.0493414,-0.0404669,-1.0174296,-0.57594883,0.24775152,0.48556384,0.02055709,0.63871384,0.13222946,-0.24304171,-0.5543615,-0.33917087,0.17576937,0.4004776,-0.35342008,0.1732417,0.41329244,-0.33031687,0.08276297,-0.49856982,-0.16362083,0.11128092,-1.349306,0.4494136,0.04769873,0.00854921,-0.04547125,-0.2375574,0.57066154,-0.27329624,0.63345915,0.26212823,-0.76913536,-0.22780412,0.24557552,-0.18287253,0.25570464,0.89756566,0.01436228,0.00603068,0.03361591,-0.898468,0.5813939,-0.08841249,-0.1262625,-0.00605271,0.30666235,0.42011732,0.6310064,-0.7559202,-0.27808923,0.2537307,-0.83045983,-0.12999615,-0.05215171,-0.02264468,0.05883318,-0.33964747,-0.11178617,0.11948799,0.209261,-0.5496212,0.57414347,-0.00921925,0.08847965,0.04592516,-0.24074781,0.3464915,-0.7887169,-0.7073391,0.07988744,0.5315157,0.35358095,0.08020461,-0.22129516,0.17290081,0.0504736,-0.08894327,-0.00620176,0.05257307,0.5016409,-0.36408517,-0.577161,-0.8663386,0.42135563,-0.3292702,-0.01840105,0.29741144,0.07685098,0.73663163,-0.24905361,-0.49479997,-0.509035,0.47502556,-0.8205876,-0.5183762,0.20303905,0.49047285,0.44658792,0.12614211,-0.3924463,-0.44975308,0.07101417,0.7553107,-0.6892747,-0.46432185,-0.02493155,0.09743762,-0.01699282,0.3170191,0.236118,0.5902151,-0.6768178,0.08673793,-0.23632318,0.03027869,-0.49375105,-0.13820373,-0.33376855,-0.5689197,0.7584243,-0.17336866,-0.82605976,0.30709103,-0.3530469,-0.43048787,-0.0306935,-0.09965517,-0.02872394,-0.12202759,0.12044877,0.28285134,0.54993904,0.31825548,-0.17328748,-0.6645016,0.10835827,0.2360643,-0.6641675,0.2868522,-0.37695688,0.2969878,0.5594272,-0.19000544,0.7002008,-0.5524048,0.48491138,-0.09867318,0.04154079,0.66359836,-0.02447115,-0.22998628,-0.47724923,0.35765377,-0.3911939,-0.09828434,-0.15728916,0.29930428,-0.04312735,0.5141893,0.10899142],[-6.28324270e-01,-1.77925631e-01,-8.61520708e-01,-2.86725432e-01,2.45009422e-01,3.55965644e-01,9.34682414e-02,-8.25778171e-02,1.12860397e-01,2.12148890e-01,1.17997754e+00,1.77013457e-01,-2.39256933e-01,-6.29048705e-01,-8.03851187e-01,-2.06865698e-01,-3.00605036e-02,-1.31937671e+00,-5.92937469e-01,-2.45478064e-01,-2.32504532e-01,-1.86896890e-01,1.13643827e-02,-4.42189425e-01,3.58190387e-01,1.37703910e-01,-2.43877679e-01,1.21188119e-01,-8.04266453e-01,6.62875772e-01,5.29602587e-01,-2.91601062e-01,-6.85958087e-01,-2.38738030e-01,3.23116511e-01,6.14840724e-02,-7.13836730e-01,1.89113081e-01,1.93483666e-01,-2.58940965e-01,5.91705978e-01,9.53129772e-03,9.57682729e-02,-7.50961006e-01,4.37652737e-01,-6.67763412e-01,4.18727309e-01,4.86821353e-01,2.10649014e-01,4.02323693e-01,2.62349006e-02,-1.47556841e+00,1.69781700e-01,-2.17676610e-01,2.30205029e-01,5.97487926e-01,-5.40146172e-01,1.39166489e-01,2.09922060e-01,-3.70799869e-01,4.33505327e-01,-2.15746388e-01,7.06305146e-01,-2.43152529e-01,3.44036013e-01,-5.40310889e-02,-9.09316987e-02,-5.20064905e-02,-1.60982326e-01,-1.10943839e-01,-9.27841738e-02,-2.15420723e-01,6.49287179e-02,3.09406012e-01,-2.02235922e-01,-2.96684593e-01,-1.09934755e-01,1.33861573e-02,4.70821351e-01,-3.11558634e-01,1.05968066e-01,2.03650519e-02,3.84489924e-01,-3.94531712e-02,-2.27777660e-01,-1.11539662e-01,-6.51500537e-04,-6.28559738e-02,1.05209880e-01,-2.08647698e-01,-6.75723776e-02,9.24386919e-01,7.51761973e-01,-4.35867250e-01,-6.26064718e-01,3.92973155e-01,1.98974907e-01,-2.22623372e-03,3.77284549e-02,-3.84151712e-02,-4.71885026e-01,8.51447344e-01,-8.88300419e-01,7.24163949e-01,-7.20989883e-01,-3.76765020e-02,5.55783629e-01,6.48239136e-01,-7.63039887e-01,9.01446491e-02,-7.31473446e-01,-1.02739990e+00,5.76704025e-01,3.65080014e-02,-4.46069896e-01,-3.30518067e-01,1.22150471e-02,4.94451039e-02,-2.13377364e-02,-4.37088721e-02,-4.08106327e-01,-4.92632866e-01,-6.53334260e-01,-4.23861332e-02,-3.60055089e-01,2.65954465e-01,4.84762728e-01,-1.07445014e+00,-1.34532914e-01,3.14144790e-01,-6.56775907e-02,2.90250093e-01,-5.68364680e-01,-2.05591068e-01,-1.32357121e-01,-3.59082460e-01,-4.65476066e-01,-8.58485997e-01,7.26056695e-02,7.15704978e-01,-9.08975005e-02,-1.87705249e-01,-3.43999535e-01,9.87095684e-02,-4.10730362e-01,-3.68016101e-02,-9.07625183e-02,1.80555075e-01,5.06849110e-01,5.69554567e-02,2.96062648e-01,-5.38354099e-01,-5.65933764e-01,-1.18276805e-01,3.11378717e-01,1.56998057e-02,6.28368631e-02,-3.30598354e-01,2.95552909e-01,-8.04044366e-01,1.23880617e-01,-5.94431341e-01,-4.93643209e-02,1.01324141e-01,2.53428102e-01,-3.69782671e-02,-3.34476262e-01,-1.62900716e-01,6.57890663e-02,-2.27555037e-01,2.02216074e-01,2.79938966e-01,7.37662688e-02,-4.68691647e-01,8.29187870e-01,-2.31900573e-01,-5.06815314e-01,2.06275374e-01,6.74975276e-01,-5.75709224e-01,-4.63223338e-01,-5.12480021e-01,-1.29520917e+00,3.25406462e-01,3.58177088e-02,2.80783713e-01,3.08938324e-01,7.02879667e-01,-2.40828663e-01,-3.43916804e-01,2.85905838e-01,-4.23200309e-01,3.34930152e-01,1.64530143e-01,-8.75997245e-02,7.13003129e-02,-8.04105163e-01,-8.45834538e-02,-4.33150291e-01,7.28080273e-02,1.90941587e-01,-1.77935556e-01,2.94849336e-01,-2.13509560e-01,1.21086739e-01,1.84051290e-01,-5.42102754e-01,1.24747917e-01,-8.17393661e-02,2.29279682e-01,-1.45231441e-01,2.99891442e-01,4.59541231e-01,-3.08097638e-02,1.35860201e-02,-2.61193097e-01,-8.67268562e-01,1.59541011e-01,-1.63387820e-01,-2.13404521e-01,1.48887873e-01,1.69174537e-01,-2.73387522e-01,3.24678421e-01,-8.54027718e-02,-1.01330006e+00,1.64784327e-01,-9.12618876e-01,-2.79790312e-01,6.67590916e-01,-8.05261314e-01,-1.38383791e-01,1.15735665e-01,-1.39942840e-01,-1.71090886e-01,5.08321762e-01,-2.03147456e-01,-2.88087904e-01,-9.49448705e-01,1.32988691e-02],[-0.15603715,-0.3115988,0.6016798,0.6286207,0.68857735,0.33695766,0.21156807,0.19672573,-0.10602822,-0.42626992,0.21462671,-1.0205109,0.07417208,-0.14955884,-0.09787784,-0.61839825,-0.6472085,0.7638579,-0.47076577,0.37513316,0.2918816,-0.93284124,-0.4647082,-0.34858826,0.25941715,-0.23960726,0.2599904,-0.29212502,0.20088974,0.3228785,0.10123134,0.5608161,0.18814062,0.3076236,0.92993146,-0.58476645,-0.21589404,0.44285265,-0.32931602,-0.5563022,0.14301781,-0.10768566,0.08431146,0.17216799,0.3702281,0.4411839,0.15657026,0.16382591,-0.13103762,0.13395816,-0.6813896,-0.45771727,0.18312569,0.08278657,0.37030527,-0.50889266,0.32644722,0.23810239,0.35205927,-0.53414994,-0.3305887,-0.38137564,-0.81114316,-0.27430028,-0.4201642,0.7468892,-0.51067567,-0.17173304,0.98224866,-0.31519404,0.03955407,0.2359718,0.00799113,-0.26141664,-0.42205545,-0.07061077,-0.15731597,0.12238798,0.46325836,-0.15476225,0.7058084,0.24847554,0.2565487,0.39702147,-0.34355745,-0.21193287,-0.30373722,0.3696948,-0.455741,-0.1660509,0.16669555,-0.60794604,-0.21485113,0.33358413,-0.582304,0.01473188,0.40546227,0.61182964,0.1446946,-0.27734867,0.20175213,0.86861354,0.00436233,0.8171295,0.6071231,-0.1718908,0.11532477,0.4669711,0.01249633,-0.00504932,-0.19166759,0.784494,0.25603533,0.2530207,0.2872751,0.00240634,0.2660691,0.39964885,-0.72254103,0.45931503,-0.7881716,-0.4289054,0.11509043,-0.49131408,-0.3125834,-0.783232,0.03038088,-0.2240577,-0.08278562,0.86706215,-0.28975087,-0.7595765,0.2954469,0.19758058,-0.09434554,-0.4272994,-0.85841125,0.09061038,-0.23327117,0.36474404,0.15946807,0.42900142,-0.11739489,0.22367539,-0.03235352,0.49473178,0.45798996,-0.04085056,0.14001805,-0.16784297,-0.24195412,-0.31870583,-0.46624687,-0.8395025,0.45012835,-0.15109962,-0.6630323,-0.04210695,-1.1640128,-0.14182638,0.70827913,0.3640308,0.21351905,-0.4894205,-0.22468543,0.4983125,-0.09072424,0.11985206,0.72519624,-0.08597805,0.15853271,-0.91960526,-0.01743699,-0.05998573,-0.21148966,-0.63583744,0.563787,0.23704423,-0.03632825,0.30471054,-0.40620944,0.36182818,-0.4420781,-0.19650428,0.0262393,-0.18849331,0.11611671,0.27450624,-0.00835636,-1.0006701,-0.57280695,-0.6890316,0.23824432,-0.10515319,-0.7213916,0.33160603,-0.02902344,0.4186534,-0.09017405,0.02537248,0.72332317,-0.6457655,-0.09581131,0.29312745,0.44244635,-0.56936026,0.1153205,0.51131314,-1.1907223,-0.56659365,0.185642,-0.21668397,0.09573951,-0.9205455,0.25407588,0.40126258,-0.08522515,-0.4254475,-0.16799717,-0.02107515,-0.18100315,-1.1107866,0.07608583,0.25500003,-0.6647649,-0.39241466,-0.04018266,-0.3802465,-0.10896395,0.24076569,0.03572404,0.220053,-0.19343878,1.0653465,0.3849424,-0.32126546,-0.55294096,0.40929413,-0.05612565,-0.1147655],[-5.56577504e-01,1.48510993e-01,4.76961404e-01,-6.03281438e-01,-4.30394769e-01,-2.35275134e-01,-2.25415945e-01,7.07756400e-01,-7.12703049e-01,-2.39790648e-01,-5.79922795e-01,7.83141851e-02,-9.00675595e-01,4.69670832e-01,6.80423975e-02,1.68369591e-01,2.56538630e-01,-1.48957920e+00,-7.83702612e-01,-1.54155403e-01,6.13866091e-01,1.42363620e+00,1.53777450e-02,-2.20996216e-01,1.01036407e-01,-3.97698075e-01,-1.02887042e-01,-9.77130234e-02,-3.58175874e-01,-1.09425254e-01,1.81142643e-01,3.11884601e-02,3.62609327e-01,-4.28380340e-01,-2.30890706e-01,-6.74339905e-02,2.21331656e-01,4.56734002e-02,4.55044769e-02,1.35522097e-01,-7.13308109e-03,3.69608313e-01,-1.72815904e-01,3.73467475e-01,9.01988074e-02,1.12911642e-01,3.95532310e-01,8.46919864e-02,4.21698004e-01,6.01735294e-01,2.31162980e-01,-6.55212224e-01,-1.16432130e+00,-2.87736475e-01,6.52560666e-02,3.21702152e-01,-1.74939394e-01,-1.07240707e-01,-9.67890441e-01,-7.71043599e-02,-6.30269051e-01,2.34439746e-01,3.75563592e-01,1.68498516e-01,3.60954343e-03,-2.71103650e-01,-3.14533174e-01,9.66757685e-02,-6.92917764e-01,2.03958780e-01,8.97639021e-02,-3.77835214e-01,1.15754820e-01,-4.55139168e-02,1.67413414e-01,-7.14613676e-01,2.70400196e-01,-8.26345384e-01,-2.11363181e-01,3.48082840e-01,3.38345021e-01,-1.71476960e-01,-1.93630666e-01,2.75199622e-01,9.37723339e-01,-5.61633408e-01,-2.75271475e-01,-4.21328127e-01,-6.62589073e-01,-4.07334685e-01,1.42950907e-01,-9.56452072e-01,-2.68063784e-01,1.30981639e-01,-7.07040548e-01,1.90890074e-01,4.11642790e-01,-1.74428970e-01,-7.13365316e-01,9.86953378e-02,-5.57085648e-02,3.52929264e-01,-5.51389297e-03,7.59334087e-01,4.38536227e-01,-9.14795578e-01,5.55056989e-01,-1.98740184e-01,-4.74352315e-02,1.63762465e-01,1.20493416e-02,1.38184145e-01,-3.84903431e-01,1.63093925e-01,-1.43591106e-01,4.19019520e-01,-4.23922613e-02,-1.16898660e-02,5.79455853e-01,6.61090553e-01,7.40548074e-02,-8.98451865e-01,-9.02550280e-01,-5.40929019e-01,4.98630226e-01,4.22405183e-01,-1.11660278e+00,1.00888705e+00,2.32378677e-01,-4.37202007e-01,1.66787028e-01,-2.08899919e-02,-6.33311093e-01,-3.93027347e-03,-8.26047212e-02,-4.27210659e-01,3.95168275e-01,-3.28251064e-01,2.79769957e-01,5.76250196e-01,-4.25441116e-01,-2.14613450e-04,4.88753505e-02,1.11461304e-01,-7.13180184e-01,1.70823991e-01,-7.57366270e-02,-9.16975513e-02,-4.93557185e-01,2.84270823e-01,3.10000330e-01,-1.25734007e+00,1.14050515e-01,-1.12552740e-01,1.58234388e-01,4.01416898e-01,4.34849590e-01,2.76329309e-01,-5.24549425e-01,-1.66777894e-01,-1.69195801e-01,5.24318159e-01,-1.74851388e-01,6.53854251e-01,1.45433433e-02,-6.59157932e-02,-6.06715083e-02,-2.78085142e-01,-3.83649826e-01,-3.75775516e-01,-7.86989368e-03,3.86626646e-02,2.49423742e-01,-2.45251060e-01,1.33316234e-01,2.37094864e-01,-1.93629518e-01,2.83797354e-01,1.86104923e-02,-2.32642204e-01,5.13114452e-01,2.74937004e-01,3.78668338e-01,1.54481262e-01,5.74718080e-02,2.66790569e-01,3.34784061e-01,-5.07676244e-01,-6.48062080e-02,-4.05389994e-01,2.53133416e-01,6.68230891e-01,-1.54463351e-01,2.77065247e-01,4.39068377e-01,-3.58862519e-01,-6.00271285e-01,-9.46269110e-02,-1.33311951e+00,3.32445294e-01,-5.52720487e-01,5.22683740e-01,-1.84347093e-01,-3.75296205e-01,-2.35437155e-01,5.23264766e-01,3.30625415e-01,-5.33753216e-01,3.74852031e-01,-4.06250775e-01,1.87603012e-01,6.65645957e-01,2.22895071e-01,-4.20403540e-01,5.56374371e-01,1.03560522e-01,1.19355526e-02,-1.13367558e+00,-4.53220963e-01,-1.26784587e+00,-3.18257570e-01,-1.01685560e+00,-6.46514535e-01,-2.42365345e-01,1.70528516e-01,-3.45500931e-02,-2.85618901e-02,4.90197778e-01,6.89330339e-01,-1.97400954e-02,8.61570910e-02,-1.52265921e-01,-3.31508219e-01,1.47453293e-01,-8.25710416e-01,1.16499513e-01,-3.56994689e-01,3.37365657e-01,1.45690054e-01,8.01126540e-01],[-2.41499379e-01,1.96854696e-01,2.47416899e-01,-1.86082989e-01,4.36884493e-01,-1.04258776e-01,1.71346679e-01,4.81240451e-01,-3.17638427e-01,-6.28392771e-03,-5.90186678e-02,-2.71379530e-01,-4.13501740e-01,-2.75238305e-01,-1.57133415e-01,-1.02625504e-01,3.13969374e-01,-2.23467678e-01,-2.75790125e-01,3.43609780e-01,6.71312392e-01,-1.98628351e-01,3.67015272e-01,4.26687181e-01,4.29485321e-01,-9.11837518e-02,2.40605533e-01,-6.53118670e-01,-3.26771826e-01,-2.89268464e-01,-7.01594651e-01,1.20349176e-01,1.00705874e+00,3.69762510e-01,4.50116932e-01,-2.05311060e-01,-5.05248785e-01,5.19841373e-01,-6.65984869e-01,3.11693013e-01,-8.76004159e-01,1.35131717e-01,3.88465762e-01,-1.35973036e-01,-3.14888023e-02,2.45728135e-01,7.45995715e-02,-2.37081796e-01,6.53898478e-01,-3.83623540e-01,-6.55500591e-01,3.98984402e-02,-6.82740211e-01,2.47627079e-01,-5.57764351e-01,-1.50312066e-01,-9.22972262e-02,-8.74816060e-01,-6.23968482e-01,8.25560465e-02,-4.79838669e-01,-5.85031152e-01,1.67412326e-01,5.68090230e-02,6.20943308e-01,8.80220085e-02,3.69181931e-01,3.43433410e-01,-1.22128360e-01,4.31564242e-01,3.67889285e-01,-4.79222596e-01,1.41201004e-01,-2.33884946e-01,-4.16073591e-01,5.45362413e-01,1.52024969e-01,3.02439574e-02,-8.72085869e-01,-5.84597230e-01,6.24317765e-01,3.65344316e-01,3.84481311e-01,2.85504520e-01,3.81143659e-01,2.34551817e-01,-2.70664513e-01,-3.21487337e-01,-6.33206606e-01,-7.52051592e-01,6.69045269e-01,-5.99704497e-02,1.26894742e-01,-2.16387883e-01,1.82824790e-01,-7.01571107e-01,5.89601219e-01,2.39660010e-01,-7.23364472e-01,6.05400652e-04,-5.21706283e-01,4.42348003e-01,-2.47291680e-02,-4.10344124e-01,-3.26674491e-01,3.23908746e-01,2.80464739e-01,-2.34149635e-01,2.47871742e-01,4.39801276e-01,-2.91219682e-01,-4.20461506e-01,6.90221608e-01,2.60275334e-01,3.60123720e-03,1.41552821e-01,3.46013635e-01,-2.83259273e-01,3.18972647e-01,-3.19447666e-01,-2.73932099e-01,-5.92201829e-01,-6.88519999e-02,6.40963688e-02,-9.50961769e-01,4.70245987e-01,-8.97847414e-01,1.49512216e-01,-5.55435121e-01,-1.01107910e-01,-7.12727964e-01,8.41513515e-01,-1.15288985e+00,-5.79531729e-01,8.48033130e-02,2.93066561e-01,3.16327006e-01,1.34406835e-01,-7.39245713e-01,4.67383981e-01,2.72519261e-01,5.21600604e-01,6.34259641e-01,5.59491292e-02,-3.08758855e-01,-3.23916553e-03,5.04003949e-02,-2.89484799e-01,-8.38304684e-03,7.07203984e-01,3.37732472e-02,-3.71487550e-02,4.84611958e-01,-2.78408557e-01,-4.60968107e-01,4.22142684e-01,-2.42445767e-01,2.83803523e-01,6.84916377e-01,-2.38334388e-02,2.53853321e-01,1.25552639e-01,-9.52375114e-01,-1.21017165e-01,-3.16219062e-01,2.17638761e-01,-1.69428855e-01,3.16398442e-01,-9.72529292e-01,1.03777342e-01,-1.53279394e-01,3.60265791e-01,1.18651375e-01,3.59215885e-01,3.57999690e-02,-7.36258805e-01,-9.77124181e-03,7.21234143e-01,1.80449963e-01,-5.26076078e-01,-1.08242679e+00,-6.83847189e-01,-3.10420096e-01,3.21802765e-01,7.01973081e-01,-3.36247310e-02,-3.94221395e-01,-1.11748266e+00,-4.70572382e-01,7.44551539e-01,-4.65128541e-01,6.93751931e-01,-5.10930240e-01,1.13834202e-01,-4.05266494e-01,-9.95138809e-02,-3.23126093e-02,4.32388157e-01,3.20742488e-01,-1.26518533e-01,-3.10262501e-01,1.73322722e-01,5.14915705e-01,9.57540989e-01,3.03275526e-01,-2.43961915e-01,-4.32125069e-02,-1.22568154e+00,3.75342607e-01,-1.38911709e-01,-3.96650583e-02,-6.34105146e-01,-5.74900866e-01,-2.58653820e-01,9.22705382e-02,-3.26223463e-01,-1.20378276e-02,-2.09760219e-01,3.68207812e-01,-2.80955017e-01,-4.34588909e-01,-3.80989999e-01,5.76591074e-01,-2.15445012e-01,2.06596807e-01,-6.04189873e-01,-5.45759857e-01,-2.49470666e-01,3.56557012e-01,9.26213339e-02,-7.26516306e-01,-1.01639792e-01,7.62881160e-01,3.18442076e-01,7.05317855e-01,-1.96233779e-01,-3.98284644e-02,-6.91014409e-01,-8.67336392e-01,-7.97919989e-01],[0.3430425,-0.2911209,0.02839902,0.74444675,0.37762913,0.37847278,-0.06698164,-0.12663314,-0.61450344,0.32912806,-0.4389453,-0.7151022,-0.38668597,-0.05261835,0.68818927,-0.19415748,-0.28725287,0.17619583,0.4778593,-0.25915527,-0.46079975,0.32546917,-0.04517109,-0.2536862,0.45040676,-0.09401967,-0.3587866,-0.17716083,-0.16997279,0.33224264,1.0355996,0.23651327,0.04554256,-0.02499145,-0.4622382,-0.03884262,-0.33248886,-0.66351557,-0.6560056,-0.17208311,-0.20513637,-0.21428475,-0.6357835,-0.16406254,-0.09217656,1.0101638,0.4304391,-0.62401974,-0.79774094,-0.72412014,-0.34464338,0.00136859,0.712612,-1.1143599,-0.27120528,0.13577361,0.109313,-0.2648609,-0.7200439,-0.41454127,-0.3711014,0.2936061,-0.5280099,-0.6257942,-0.33541587,0.03409524,-0.05269879,0.4880472,-0.41354775,-0.46880493,-0.45633656,0.2967339,-0.6696081,0.20498937,0.51509726,0.43292314,-0.61113137,-0.5823481,-0.7411106,-0.05975483,0.11779989,0.2268471,-0.01098834,0.05596415,-0.0040559,-0.02367617,0.4519011,0.1979816,-0.4235433,-0.3217146,0.0175055,-0.05761493,-0.310196,-0.16592276,-0.5587189,0.40560022,0.31211764,-0.15931615,0.03766758,0.00872939,-0.25074768,-0.42422736,-0.58780104,0.05164286,0.53576654,-0.0936624,0.16896029,-0.29201302,0.14925572,-0.33233574,-0.6781345,-0.52741075,0.29116362,0.22883496,0.17549828,-0.3278208,-0.05874048,0.49129927,-0.50049174,0.14326279,-0.5052557,-0.2023481,-0.09970702,0.34264022,-0.25777483,-0.22924367,0.12876514,0.41468024,-0.1624354,-0.14106494,0.52673423,0.3152776,0.20623146,-0.45082617,0.41119096,-0.24429291,-0.2261175,0.12271525,-0.16879377,0.19128501,-0.23997962,0.35154906,0.35198388,-0.17926073,-0.3985668,-0.5348957,0.7247355,-0.12197914,0.01157244,0.8381709,-0.40834507,0.1104655,-0.77449346,0.1093148,-0.5092712,-0.2840944,-0.13710992,0.59303766,-0.53370965,0.3163498,0.08055435,0.5454722,0.34153026,0.26860246,0.42492676,0.13258284,0.14544787,0.2702985,-0.2391113,0.19764051,-0.15715702,0.37553602,0.31016755,-1.013513,0.8447594,-0.55316955,0.32631642,-0.31510502,-0.29222113,0.5286652,-0.8624877,0.19011727,-0.08210186,-0.19215713,0.6524062,-0.5478604,-0.49556166,0.0691676,-0.02839465,-0.02546953,-0.6944926,-0.35297912,0.09454322,0.5157737,-0.48840627,0.04742629,-0.08580615,0.14120334,0.21283719,-0.00488089,0.37483373,0.6107986,-0.03090683,0.548985,-0.7616673,0.6353877,-0.10389343,-0.24760023,-0.3447186,-0.6208945,0.6957758,-0.48704994,0.16467907,-0.32253045,0.23848614,0.22685225,-0.33476752,-0.30455822,-0.05630805,0.44004855,-0.60552675,0.05017036,-0.06489718,0.03232434,0.48078266,0.01687998,0.08760344,0.06897004,0.43424213,-0.0503012,-0.9688679,-0.12902492,-0.1741567,0.5718297,0.76383525,0.10355141,-0.604228,0.19745794,0.39236215,0.1023305],[0.60214734,-0.5737601,0.35626546,0.5627827,0.5851186,0.19640274,0.00400168,0.35182694,-0.02611874,0.9171702,-0.48109233,0.31054005,-0.08846554,0.60188293,0.23916231,0.49652192,-0.39943558,-0.5653107,0.08725721,0.5468239,-0.29890335,0.3853165,0.23639123,0.7843665,0.33012107,-0.2896022,-0.02600428,-0.5438043,0.2960538,0.0120219,-0.23637663,0.7995568,-0.11783488,-0.5807036,-0.21533206,-0.11257803,-0.09661614,-0.28365308,-0.39805022,0.07318919,-0.24474166,-0.27502814,-0.4269423,0.34239638,-0.09457762,-0.65906554,0.16030964,0.09756051,-0.12869129,0.14562368,-0.23959501,0.04828146,0.29666758,-0.23887224,0.32929704,-0.20480217,-0.14325282,-0.3011964,-1.0381036,0.12338388,0.1311799,0.11562788,0.41262713,0.5094575,0.2549035,0.08270971,0.668386,-0.63631976,-0.29011,-0.5812092,0.08482933,-0.08982395,-0.7237068,0.37358063,-0.2567305,-0.6068274,0.5010475,-0.03696192,-0.04527084,0.1767356,-0.7070355,-0.10511191,-0.23490243,-1.0823616,-0.12121093,-0.05916023,0.21278206,0.4179193,-0.7855447,-0.9223868,-0.09524918,0.0701171,0.28577235,-0.1262926,-0.19812536,-1.2098423,0.01156177,0.91081184,0.5541225,0.29735452,-0.14276221,0.5038611,-0.09062682,-0.0599553,0.25653267,-0.258601,-0.1738761,0.22195826,-0.27912527,-0.23602425,0.20303185,-0.5178301,0.01106543,-0.44719216,0.06023201,-0.7299352,-0.45687425,-1.3694035,0.02741915,-0.21777067,0.31222078,-0.28454396,-0.35761166,-0.21302806,-0.07527515,0.32957506,0.3027735,0.59742963,0.34322247,-0.04236805,0.22129089,-0.8804082,0.42805383,-0.42887148,-0.3134215,0.05368628,0.11319923,-0.5689093,-0.8803607,-0.30644915,0.45521188,0.38302672,0.47648516,-0.6241196,-0.02531718,-0.16671738,0.29411566,-0.53775144,0.302447,-0.55692667,-0.37791607,0.08396952,0.8210104,1.3051199,0.6729805,0.08128282,-0.40740246,0.57239,0.5637472,0.38366237,-0.74381304,-0.6150213,0.20835905,0.48789784,0.4439599,0.41087538,-0.49749798,-0.09740985,0.2566358,0.48403454,-0.18713945,0.33259735,-0.22899629,-0.5758919,-0.16612522,-0.28434324,-0.3215712,-0.11396901,0.2902947,0.19709297,-0.56245005,0.5774645,-0.05617276,0.38263786,0.13976969,-0.08880194,-0.02758733,-0.24676953,-0.2539423,-0.14090551,-0.38891277,-0.39032686,0.04541161,-0.4775458,0.6536377,-0.09543522,0.23243894,-0.5168626,-0.4739536,0.2049638,-0.5855378,0.06255466,0.26828712,0.10655588,-0.09510672,-0.6950517,0.07592326,0.0053581,-0.06414479,-0.4491612,-0.28081673,0.10914876,-0.20201032,-0.5349737,0.29861856,-0.9505935,-0.03577428,-0.5562447,-0.1217979,-0.01103035,-0.6361625,0.07227143,-0.26422796,-0.90274566,0.7282233,0.23324442,0.14345124,-0.00573759,-0.04419891,0.8020106,0.01109822,0.13820197,-0.32502976,0.49640474,0.13866284,-0.55493647,-0.42389297,-0.25208175,-0.5181364,0.3011742],[0.22765483,-0.11397344,-0.40203905,0.4836597,0.6268686,-0.409786,0.48542586,0.32396197,-0.5633323,-0.2727837,-0.43819657,-0.31812948,0.00584434,-0.07303539,0.20605604,0.27256337,0.36769295,-0.5737674,-0.28543407,0.57313496,0.5778832,-0.43649763,-0.04067355,0.61338264,0.23699644,0.10821345,-0.22314225,0.07228525,-0.25107265,-0.47343266,0.04879817,-0.07086536,0.6527049,-0.19307254,-0.1090642,-0.30474076,0.4269108,-0.43989664,0.0989065,0.38163722,0.08319595,-0.84258455,-0.3883946,0.19742565,0.05849363,-0.3036955,0.20439449,0.09772518,0.28458494,-0.76033753,0.2416333,-0.36349872,0.0918311,-0.20611265,-0.15241127,-0.0344996,0.4844047,-0.79747516,0.45745528,0.4207199,-0.38499802,0.14217721,0.11147096,-0.0459492,0.5998279,0.09789472,0.7153614,0.05845045,0.15540442,-0.44443798,0.15309191,-0.4666524,0.55394596,-0.15057932,0.07745934,0.19400503,-0.38735095,-0.2115249,-0.42720857,0.0075005,0.2720673,-0.07072552,-0.18913305,-0.33695003,-0.07425606,0.4784541,0.2694078,-0.13230036,-0.34693927,-0.37703937,-0.14171386,0.58193386,-0.40878087,-0.32399112,-0.07590785,-0.27895373,0.21878757,1.0675741,0.18501589,0.03232991,0.41705245,-0.3146549,0.5307484,-0.6005658,0.22096285,-0.29915816,0.46081612,0.40043834,0.9963087,-0.13153926,0.2221564,-0.674174,-0.10054661,0.0376934,-0.20106481,-0.0573863,-0.46598753,-0.2696277,-0.13474204,0.7110062,-0.04151237,0.5033502,-0.02423815,-0.58721906,-0.47036427,0.42141283,0.47045746,-0.27216983,-0.5488881,0.2106417,-0.21278793,0.02118643,-0.5446556,0.44234842,0.39481202,-0.06952364,0.49454537,-0.2865852,0.02798529,0.6573001,0.4211232,-0.31507325,0.86254615,-0.32432818,-0.09077255,-0.24659573,0.05214315,-0.81755954,-0.20255244,-0.739549,-0.8761145,-0.06799197,-0.3356708,-1.1856022,-0.1733786,-0.38043308,0.24769564,0.4862692,0.02913124,-0.5487673,-0.11203461,-0.26443642,-0.11945587,-0.66799074,0.2091478,-0.02728401,-0.4272916,-0.4287072,0.48786053,-0.81251013,-0.04679364,-0.2512105,0.521918,0.03452305,-0.12305436,0.78544223,0.5827887,0.09120562,0.01865612,-0.35254094,-0.22949319,-0.13858806,-0.5491576,0.16673936,-0.5048687,-0.04569959,-0.29956353,0.34714562,-0.22145817,0.5970494,-0.6374784,0.5475449,-0.51230454,-0.34657988,-0.42212132,-0.07739846,0.04904991,-0.41543505,-1.0314441,0.44294864,0.51363814,0.42653233,0.36182296,-0.28563163,0.08840279,0.1052089,-0.7489224,0.2139935,-0.5635366,0.23182143,-0.22482415,0.0024938,0.25336725,-0.13471839,0.29909936,-0.4409691,-0.52714884,-0.7061223,0.20479345,0.26461273,0.30003008,-0.0402481,-0.05711986,0.44115624,0.43940768,0.08010611,0.46176523,0.03055261,-0.14034764,0.25548443,0.0432259,-0.13119197,-0.5777697,0.45007437,-0.43971777,-0.32901213,0.13551119,0.37423468,-0.42216787,0.3306098],[5.37615716e-01,1.89511925e-01,1.12422913e-01,5.91431081e-01,8.17250133e-01,-5.59974551e-01,2.24546686e-01,-3.12068373e-01,7.88595080e-01,-8.91919658e-02,3.37685555e-01,2.41273432e-03,-6.25308812e-01,2.02828810e-01,-1.04467237e+00,4.80373353e-01,2.10605860e-01,3.05239558e-01,-1.81053221e-01,-6.88285828e-01,1.01154363e-02,-1.22997153e+00,-8.08764398e-01,2.34005094e-01,3.18246454e-01,1.90186575e-01,-4.25939076e-02,-1.03417948e-01,2.65146911e-01,3.94059755e-02,-5.38484693e-01,1.23175591e-01,1.36849955e-01,1.25117421e-01,-1.29156423e+00,1.29085511e-01,1.83547363e-01,-1.58153296e+00,-6.43832907e-02,-1.07449815e-01,-1.93577707e-02,-7.44928062e-01,-5.52563705e-02,-4.18753952e-01,-1.69752359e-01,6.41614676e-01,-1.21745773e-01,-1.02498621e-01,4.70778167e-01,-9.34843197e-02,-6.50944531e-01,-1.68682545e-01,6.42386556e-01,2.05589414e-01,3.97768080e-01,-8.42070401e-01,-6.63835704e-02,-1.20556191e-01,2.93905199e-01,5.40567398e-01,1.13050476e-01,3.45700622e-01,-4.10008162e-01,9.06143039e-02,1.75614700e-01,3.47130835e-01,4.30676937e-01,3.28373700e-01,-5.88973641e-01,8.57482493e-01,-2.64434546e-01,-4.48592693e-01,-5.01101494e-01,-2.82524526e-01,6.70518935e-01,-1.00082934e-01,5.26599772e-02,-5.00064762e-03,-5.21544755e-01,1.04497723e-01,-5.85494459e-01,7.02665970e-02,5.43334305e-01,-3.81145716e-01,-1.07285023e-01,-7.05136955e-02,1.99568570e-02,5.24431050e-01,2.97091633e-01,-1.01851821e-02,-1.14317429e+00,-3.92231941e-01,4.40210760e-01,-5.39343655e-01,-4.78113055e-01,-2.34818459e-02,7.15674460e-01,1.48708199e-03,-4.17223632e-01,3.86226773e-02,-3.74471277e-01,2.66349077e-01,2.85523832e-01,-4.30609956e-02,-3.98244172e-01,-2.96207428e-01,-2.14860529e-01,3.90810966e-01,-5.77126205e-01,3.41247976e-01,-8.25628519e-01,9.71833318e-02,3.70084435e-01,3.04187804e-01,-2.78853387e-01,-5.74846804e-01,-3.65602881e-01,-2.74528861e-01,-3.25337410e-01,1.14123613e-01,-2.15966836e-01,8.42739865e-02,-1.23083137e-01,1.60797462e-01,-8.63358080e-02,-2.74409503e-01,1.11930713e-01,-1.00962353e+00,1.90134466e-01,5.24536312e-01,5.86436510e-01,3.07544738e-01,-3.82320821e-01,-2.18983442e-01,1.37572721e-01,-2.18152747e-01,2.55628914e-01,-6.11557186e-01,2.25864634e-01,-6.69303060e-01,3.28151107e-01,1.48332492e-01,1.97291330e-01,3.01816940e-01,-4.18821961e-01,-2.08844617e-01,-3.62615675e-01,6.95992351e-01,2.52838045e-01,2.84013093e-01,2.78072413e-02,9.85444635e-02,6.21805638e-02,2.10291609e-01,1.89562902e-01,1.96756989e-01,2.48525575e-01,-3.74961495e-01,9.97103572e-01,-4.97573227e-01,-5.35369933e-01,4.85665292e-01,5.70766628e-01,-2.51300424e-01,-1.74207643e-01,-7.05867946e-01,-3.51261348e-01,-1.74525268e-02,1.43705130e-01,-2.43792430e-01,-3.67018044e-01,7.03558028e-01,3.54020983e-01,-2.68672228e-01,1.91425368e-01,3.30991983e-01,-1.26455858e-01,-1.67573467e-01,-4.73903259e-03,8.43366012e-02,6.19980633e-01,1.18839294e-01,3.99219066e-01,-1.57909617e-01,-3.95649940e-01,3.44884098e-01,3.65541160e-01,1.43060148e-01,-8.94213021e-01,-1.36151314e-01,-5.09840369e-01,-1.66069627e-01,1.45691380e-01,-3.15423578e-01,3.11795652e-01,-4.76388097e-01,-4.72360581e-01,-6.01223588e-01,8.17107201e-01,3.89743894e-02,-1.72817200e-01,5.94417453e-01,-2.56444573e-01,2.01003745e-01,-9.99697983e-01,5.88215403e-02,1.03531308e-01,-1.30012661e-01,-5.38517535e-01,2.02835873e-01,1.09583735e-01,1.15063772e-01,-6.48256838e-01,1.11920297e-01,-5.24093032e-01,2.84579307e-01,6.97938681e-01,-5.12075961e-01,-9.32789072e-02,1.92366362e-01,-1.25524044e-01,-5.51469922e-01,-1.07736319e-01,6.89134300e-02,-5.66749156e-01,-5.32978594e-01,-3.48857552e-01,4.98389482e-01,8.19169879e-02,-4.20432910e-02,-4.10726905e-01,-1.85360566e-01,-3.49436849e-01,1.03569645e-02,-5.54115415e-01,-1.49449408e-01,5.98121226e-01,-3.02926619e-02,3.13678056e-01,8.50364983e-01],[-2.61721253e-01,3.08633626e-01,-1.75203100e-01,-3.74963641e-01,-2.65212774e-01,-9.24778804e-02,2.49972939e-01,6.28277481e-01,-6.77839279e-01,-4.80016805e-02,-5.20629346e-01,2.74959137e-04,1.13109834e-01,5.28135598e-02,8.97129893e-01,-2.04651937e-01,-4.10451263e-01,-8.51447433e-02,1.30233198e-01,9.90313366e-02,-3.66042584e-01,-1.51816130e-01,2.84937412e-01,1.84719384e-01,-6.54929042e-01,-1.10654339e-01,6.05888188e-01,-8.91842008e-01,-7.53353834e-02,-2.09196255e-01,9.70480263e-01,-6.81875527e-01,2.46596381e-01,-3.96781750e-02,-3.49992365e-01,-1.85470898e-02,2.10399628e-01,-1.01323724e-01,2.67036408e-01,6.10298291e-03,-7.04899371e-01,4.31357265e-01,1.11811170e-02,5.19660473e-01,2.10460350e-01,2.08242804e-01,-6.01010561e-01,-7.79568493e-01,-5.79966307e-01,3.45013551e-02,-1.78096369e-02,1.07935563e-01,1.85277611e-01,2.26374473e-02,3.93724293e-01,-3.94358635e-01,9.33213830e-02,-5.57585359e-02,-2.18361706e-01,5.30308187e-01,-3.99219066e-01,8.37879002e-01,-4.50885266e-01,6.01177871e-01,-8.53701651e-01,6.71133935e-01,3.11514050e-01,3.36314201e-01,-2.80674174e-02,-4.75559086e-01,3.68005991e-01,-1.91923603e-01,-3.08966130e-01,-9.08486545e-02,2.31214538e-01,2.82297313e-01,-9.45645794e-02,4.06521499e-01,4.53287968e-03,-1.19830990e+00,-1.53402984e-01,-3.57862748e-02,4.08641487e-01,-2.92600542e-01,6.96267068e-01,1.17758811e-01,3.62022817e-01,-4.10409361e-01,6.55476078e-02,-2.02147037e-01,-2.97994524e-01,-5.15440404e-01,-1.47176787e-01,-1.62510555e-02,4.47472557e-02,-8.06155980e-01,2.90490299e-01,-4.50875700e-01,-6.56361803e-02,-6.50145829e-01,-1.82790563e-01,1.01616991e+00,-2.66360432e-01,4.42084193e-01,-1.35320038e-01,-2.64069110e-01,4.09901470e-01,1.94446698e-01,2.49848485e-01,3.62172604e-01,3.67523193e-01,-6.27269983e-01,1.91692635e-01,-4.67236221e-01,-2.76661664e-03,2.05230236e-01,2.78405786e-01,2.21600518e-01,1.98489144e-01,-6.80450797e-01,1.03429958e-01,-1.80895889e+00,-6.10592067e-01,1.15326196e-01,4.67921011e-02,5.45570195e-01,4.51068282e-01,-3.39973807e-01,1.75433740e-01,2.33085141e-01,1.53409094e-01,8.67773965e-02,1.87555939e-01,1.49551118e-02,-1.66337162e-01,7.39757791e-02,-1.23329267e-01,-1.35304344e+00,-2.06557542e-01,1.12929448e-01,4.53709990e-01,-2.05147937e-01,-1.41304031e-01,-4.49893653e-01,2.24469855e-01,-9.87366661e-02,-1.05645776e+00,5.36768973e-01,-4.29719299e-01,2.43686959e-01,3.21858317e-01,-2.18332618e-01,5.46687245e-01,-2.43377641e-01,5.67480206e-01,-2.89081577e-02,6.04252994e-01,-5.70524991e-01,-1.25737250e-01,-4.54932541e-01,-2.56596148e-01,-2.30337694e-01,2.20297754e-01,-3.79051507e-01,-1.49167880e-01,2.09283933e-01,-2.35155150e-01,-5.00721991e-01,5.24683416e-01,3.15430999e-01,-2.31523618e-01,-3.67738307e-02,4.43981409e-01,4.38552707e-01,2.06259534e-01,-9.59198415e-01,-5.75558245e-01,5.55681229e-01,-4.78279710e-01,4.35612015e-02,-4.84323353e-01,6.89295709e-01,-2.84878761e-01,1.62469551e-01,1.35937020e-01,1.06392957e-01,2.62185305e-01,-9.80076134e-01,-9.30437297e-02,-5.09873452e-03,-9.11367312e-02,6.73176706e-01,-2.73371667e-01,6.14229310e-03,-2.20509514e-01,1.49835736e-01,4.09613013e-01,-1.70924783e-01,8.07311237e-01,2.18202278e-01,4.76687759e-01,-4.12028104e-01,-3.38364869e-01,-1.29668570e+00,2.02930629e-01,3.30982327e-01,3.13069224e-01,3.37199301e-01,-1.38000790e-02,2.87505656e-01,-5.12805164e-01,7.87501276e-01,-3.06523234e-01,-4.29200709e-01,-3.00652415e-01,1.62368700e-01,-1.85852706e-01,3.84467244e-01,-6.79330170e-01,-3.74670029e-01,-9.33265910e-02,2.40067407e-01,-6.79731548e-01,-1.87916145e-01,3.29536766e-01,6.28060639e-01,2.66595274e-01,8.37270543e-02,1.13476627e-01,4.56057414e-02,2.69841384e-02,2.68892258e-01,5.05211890e-01,-2.39345446e-01,2.57109463e-01,5.58875859e-01,1.52914226e-03,2.64100909e-01,-1.47092834e-01,1.88484609e-01],[-0.10452706,-0.44357812,-0.05162983,-0.86029404,-0.5362544,-0.08621196,-0.28866306,-0.09704888,-0.00492039,-0.11932129,0.11371712,-0.15787742,-1.61642,-0.17807661,0.33088294,0.12713817,-0.41330302,-0.28401086,0.14340156,-0.01365312,0.13101654,-1.4700834,-0.07291763,-0.54453504,-0.19694409,-0.01318859,-0.18250002,-0.46479344,0.5022605,-0.04449859,-0.02676368,0.44353005,0.07200059,0.09216909,0.24102435,0.59872526,0.5682682,0.10258409,0.5680381,0.03164722,-0.5176233,0.57102257,-0.75130695,-0.01745329,0.01062648,0.5316731,-0.39590624,-0.04166355,0.46292245,-1.0923735,-0.3601995,0.30209202,0.02249281,-0.7641762,0.28421295,0.18398128,-0.09932374,-1.5656543,-0.7851512,-0.5656613,-1.0807891,0.03357816,-0.0586159,0.15519516,-0.4196155,-0.12046237,-0.37064698,-0.02350012,-0.2634184,-0.05902833,-0.10967404,0.37768438,-0.25341436,0.2922172,0.47886163,-0.16365734,-0.25530913,0.06750516,0.29305762,0.0720755,0.09538624,-1.4109838,0.26741964,0.03304471,0.30682525,-0.07951851,-0.05625282,0.12332488,-0.01266137,0.37732455,-0.49590772,-1.0208206,0.20383129,0.21557687,-0.47972086,-0.01531059,0.21353261,0.09175199,0.10031234,-0.3929398,0.14761716,-0.25547156,-0.12302162,-0.08383201,-0.554916,-0.5716628,0.26531774,-0.1464903,0.3673461,-0.2969336,0.7757362,-0.14961822,-0.399695,-0.66039664,0.49995175,-0.34880307,-0.59946215,0.9345544,-0.38330275,0.07312345,-0.46060625,-0.28988633,-0.50213534,0.9211446,-0.8103225,-0.2718789,0.3216556,-0.54545856,-0.0962384,-0.32109028,0.0799144,0.22816846,-0.24132785,0.16356239,-0.29162428,0.2734309,0.3396376,0.34288275,-0.20272635,0.0614307,0.10931914,-0.38555646,0.32765245,-0.07053898,-0.04508838,0.19401275,-0.20077422,-0.07130557,0.32586637,0.5021269,-0.37277204,0.7914819,-0.01565903,-0.3361266,-0.3225976,0.2089679,-0.8244442,0.44595897,-0.23434053,-0.10762195,-0.59615016,0.4384806,0.68926907,-0.6119474,-0.03334461,-1.3439548,0.1910797,-0.23041143,0.2991494,0.24408746,-0.6241673,0.3068018,-0.34568235,0.24602701,0.65739477,-0.04692873,0.7402888,-1.4563171,-0.07620781,0.1426821,0.24542357,0.33376238,-0.28628156,-0.61878085,-0.27685657,-0.8403102,-0.21576175,-0.34186465,0.5534768,-0.51541066,0.4951572,0.01716785,-1.0588043,-0.01504314,-0.25221342,0.14374796,-0.16994298,0.09898492,0.33940148,-0.08151025,0.02311669,-0.15580739,0.05221433,0.02596304,-0.06263322,0.07287546,0.5513805,0.07257495,0.8144066,-0.16025712,0.2574518,-0.10780686,0.0971587,0.56830305,0.3509932,0.36724293,0.14791572,-0.5176427,-0.01493227,0.46544912,-0.5231792,0.06945104,0.48600173,0.11786618,-0.34601435,0.8429039,-0.08363806,0.01992025,-0.38051474,-0.36569124,0.60310495,-0.41659477,0.31588647,0.38458374,0.36641395,-0.2904237,0.4295237,-0.17580439,-0.34458017,0.02506181],[0.6616067,0.91428685,-0.5127075,-0.6158166,-0.90601945,-1.0142522,-0.11920504,-0.02333417,0.18951632,-0.12186079,0.3288173,0.9307113,0.07585886,0.7858484,-0.09852835,0.09463204,0.5320413,0.6020886,-1.1594386,-0.38986632,-0.13635488,0.44987643,-0.71205807,0.6829062,0.53963155,-0.4662089,-0.25167558,-0.44147468,-0.330756,0.41422826,-0.30061248,0.19651559,-0.29338744,0.18221484,0.5934514,0.46777925,-0.81573147,-0.13545525,-0.05961706,-0.43457276,0.28623298,-0.10854008,-0.35529926,-0.65396786,0.13447076,0.44086057,-0.07568517,-0.44147608,-0.00192868,-0.5530547,-0.53244936,0.73855484,-0.9924686,-0.38510245,-0.35600144,0.3840482,-0.28645888,-0.00398889,0.26557726,0.32717586,0.06011399,0.09345451,-0.749996,-0.42538723,-0.19394167,-0.3973464,-0.24401203,-0.50445044,-0.12419201,-0.43979743,0.0071691,0.3098946,-0.22452912,-0.10574132,-0.15447389,0.67326975,0.03338511,0.7130291,1.1051308,-0.39121163,-0.6733531,-0.7319809,0.34831107,-0.43715397,-0.24541718,0.20813082,0.48720038,0.09826235,-0.51189953,-0.06372438,0.3598141,0.3151277,0.36895096,0.58987796,0.6665622,0.41296098,0.34617987,0.1356589,0.07240814,0.4939968,-0.5200265,0.12538233,-0.10187831,-0.294452,-0.3323603,-0.9256932,-0.486785,-0.39338577,-0.9637951,-0.3250833,-0.10348068,-0.01676308,-0.43388024,0.16716686,-0.29361096,0.20537682,-0.5015363,-0.61961687,-0.64231783,0.37161246,0.40039307,0.6472683,-0.8325624,-0.45653704,-0.04258712,0.6836507,-0.15744305,0.11198556,-0.5678433,0.12632515,0.40263104,-0.46482506,0.00148919,-0.00747839,0.9809026,-0.1630248,-0.29273298,0.16507861,-0.7762903,-0.35651606,-0.13383017,0.96912855,0.525713,-0.02076105,0.11018188,-0.03921937,0.00185251,0.01415034,0.07368261,0.03398828,0.14938457,0.01462698,0.10615202,-1.2554463,0.21966088,-0.5083012,-0.13445371,0.59385246,0.24802431,-0.5770548,0.45533243,-0.04261157,0.49967846,0.04796259,-0.48073262,-0.59803206,0.3968353,-0.09287576,0.2052622,-0.00997773,-0.35982007,-0.00779842,-0.04833409,0.4801464,0.16953011,-0.5305511,0.20265281,-0.28210545,0.02707285,-0.12289824,0.4752089,-0.05691059,0.6130356,0.11785647,-0.7378482,0.30767518,0.4946279,0.16579776,-0.24589019,-0.782327,-0.5410471,-0.04759906,-0.08401921,-0.17623085,0.09387454,-0.0213626,0.4096492,-0.89850277,0.23213826,0.4235476,-1.0498476,0.38073587,-0.12936269,0.64792347,0.14695622,-0.3546383,0.2773405,0.19184797,-0.0343722,-0.21828842,-0.38690796,-0.34305382,0.31294858,-0.27419958,-0.03643838,-0.3018975,0.20416929,-0.4708575,0.20739612,0.8005499,0.2544351,0.07626476,1.0051696,0.26456314,0.8100149,-0.2447576,0.35822868,0.68142647,-0.00977824,-0.5653626,-0.03263413,0.29537123,0.1673314,0.01764651,-0.07406889,-0.2879324,-0.2569183,0.12531535,0.00486198,0.13528638],[1.60513565e-01,-1.85334861e-01,6.29182681e-02,1.09136605e-03,5.26835680e-01,-4.53521490e-01,-3.64396960e-01,2.19991982e-01,-1.79932505e-01,2.53442287e-01,7.29185268e-02,-8.72690007e-02,6.51085228e-02,4.85085726e-01,-5.85400164e-01,2.94475704e-01,-3.59054208e-01,2.37961292e-01,3.38845909e-01,-2.96732217e-01,-2.91887641e-01,-1.03016615e-01,-7.31108069e-01,2.14164883e-01,2.98969924e-01,4.39451367e-01,-2.53422648e-01,5.93645155e-01,-1.02140129e+00,7.85794035e-02,-4.34876025e-01,-2.33086944e-02,4.84914452e-01,-7.75450468e-01,-7.52020106e-02,2.18752384e-01,2.04162359e-01,-5.08289456e-01,1.58884823e-01,8.01199600e-02,-4.30989750e-02,-2.77377963e-01,1.08688973e-01,-7.98762590e-02,-3.21266979e-01,4.38836077e-03,2.71813482e-01,4.31071311e-01,-7.55927190e-02,2.48636961e-01,9.48856622e-02,-3.63102276e-03,1.58928201e-01,-9.04492378e-01,-4.55561057e-02,4.22121912e-01,-7.65244901e-01,3.39289069e-01,-5.49148619e-01,-1.76990211e-01,9.37329456e-02,-3.92918676e-01,-1.15825415e-01,3.33050579e-01,-2.93957502e-01,6.12853765e-01,-2.87941456e-01,5.52662492e-01,-1.88403264e-01,-4.24130350e-01,-2.78652400e-01,6.89742804e-01,-7.50409067e-02,3.22491266e-02,3.00422847e-01,1.06615655e-01,5.39457142e-01,-1.44927815e-01,2.95462348e-02,-4.14158165e-01,6.34216368e-01,5.61633945e-01,1.30972996e-01,6.15985692e-01,-3.23068380e-01,-2.72232592e-01,-1.69233918e-01,-4.44619894e-01,7.76372910e-01,3.30053478e-01,3.44109684e-01,-6.56400323e-01,-5.95687851e-02,-3.70861888e-01,6.21248543e-01,1.60362095e-01,-6.81820631e-01,9.85077560e-01,5.45942128e-01,-1.03657925e+00,1.03467070e-01,-5.88436544e-01,7.29798496e-01,4.57198173e-01,-8.54225606e-02,2.46788874e-01,2.93300033e-01,1.45985261e-02,-8.85074437e-02,1.77561536e-01,-5.23990631e-01,6.60584152e-01,-4.17404771e-01,5.02965987e-01,1.29773989e-01,3.51606518e-01,3.07116419e-01,9.72697318e-01,-5.85599184e-01,1.78494379e-01,-2.08817888e-02,-1.01793468e-01,1.15985405e+00,-1.24323867e-01,-3.80546629e-01,3.53107482e-01,6.72981858e-01,2.53038317e-01,-8.02649930e-02,9.52548385e-01,1.08297095e-01,2.94703782e-01,4.05315161e-01,-4.10447001e-01,-6.10704780e-01,-4.64296699e-01,5.31776071e-01,-1.80120051e-01,-8.19210410e-01,-3.75828117e-01,3.50843766e-03,-4.21599656e-01,-2.01615468e-01,1.93751052e-01,5.12040436e-01,3.70479167e-01,6.18408388e-03,4.01094705e-01,1.53248131e-01,-2.08959579e-01,-4.91321713e-01,9.62450877e-02,1.47951365e-01,-3.32006663e-01,1.30354375e-01,3.68676692e-01,6.71644747e-01,-6.18954301e-02,-6.89991610e-03,-5.92291594e-01,-5.20218372e-01,1.38978750e-01,-6.32511914e-01,1.72240406e-01,-8.33467722e-01,8.17542672e-01,-5.11569560e-01,5.62155955e-02,6.03517175e-01,-1.94930956e-01,-5.72295725e-01,-9.42139886e-03,1.01072557e-01,-7.06846491e-02,-7.30917990e-01,4.75694507e-01,-3.17956448e-01,-5.39312720e-01,-2.55983055e-01,-2.67888028e-02,-7.81491250e-02,-2.98096716e-01,-1.54295772e-01,6.42793536e-01,-2.02938825e-01,7.33841836e-01,-1.63456321e-01,-3.63459177e-02,-8.92441571e-01,2.73772717e-01,8.73815268e-02,7.86505118e-02,1.61830455e-01,-1.25277743e-01,1.70687586e-01,4.15640265e-01,-7.38733649e-01,-3.46591830e-01,-5.23383953e-02,-6.51918054e-01,-2.22636938e-01,-6.46589249e-02,-2.17086926e-01,-1.22255392e-01,1.97408766e-01,-2.11469397e-01,-7.67007098e-02,-6.57805085e-01,6.37307703e-01,-1.75436601e-01,5.49356103e-01,-4.77326274e-01,-2.50261635e-01,1.97871074e-01,1.55820116e-01,-1.09320432e-01,-2.92859197e-01,3.13499600e-01,5.81715584e-01,2.63446897e-01,-5.91726959e-01,1.44638717e-01,1.37739226e-01,2.27771886e-02,5.02494574e-01,-1.44356385e-01,-5.60826004e-01,2.22628087e-01,6.70314133e-02,-8.07533562e-01,-2.41466343e-01,2.39641339e-01,-6.92737401e-01,-1.03185594e-01,-9.02354419e-02,-1.17147855e-01,-8.12787935e-02,2.38222629e-02,1.27422616e-01,-3.45108695e-02],[-0.02485669,0.380621,0.01787959,-0.39203522,0.48212782,0.09504802,-0.3229376,-0.41534752,-0.4263078,0.21349806,0.305489,0.2132812,-0.46302328,0.06200241,0.13572986,0.09470335,-0.60911506,-0.09334892,-0.11546892,0.10293039,0.10103153,-0.4092322,-0.08593746,-0.75380313,0.03496397,0.12865691,0.14534576,-0.06234893,-0.33999595,-0.6378125,0.09825496,-0.05197421,0.01559881,0.31164187,0.61428094,0.16649465,0.35869494,0.5640697,-0.2986383,-0.28263837,-0.82052886,-0.04053642,-0.43747506,0.5868996,0.19797617,-0.47585696,0.5936818,-0.30138278,-0.46290234,-0.28737694,-0.25997192,-0.11250543,0.12640156,0.07796185,-0.13291365,0.4887769,-0.481782,-0.12665434,-1.3363914,0.04617102,0.01272895,0.2962243,0.24335551,-0.33948633,0.15972653,-0.6545288,0.20876785,-0.6926615,0.08809546,-0.4320803,-0.07090379,-0.66436905,0.5285341,-1.1260227,0.0349365,-0.30731297,0.35481837,0.7514175,0.18508944,-0.47324887,0.4628414,-0.40238804,-0.40255937,0.16929777,0.9932272,-0.06631253,-0.33080366,-0.03506042,-0.20003003,-0.18202588,-0.18018845,-0.24855462,-0.6811314,0.29760066,0.25762993,-0.0515075,0.1071502,-0.3213374,-0.30124524,-0.03115776,0.17198817,-0.01303739,-0.12168096,-0.04621495,-0.27747083,0.4515045,-0.21409947,0.00626309,-0.53191066,0.806874,-0.32523277,-0.3731274,-0.03310235,0.39401227,-0.1693187,0.05112809,-0.3340518,0.5303938,-0.18869911,-0.8989619,0.0970078,-0.94454336,-0.7100596,0.18019728,0.34081474,-0.21386576,-0.5022024,0.28307042,0.2695366,-0.28903145,0.19555838,0.05296295,-0.0257827,0.06806974,0.17463957,0.14262441,0.24885133,-0.3573462,0.777731,0.3642095,0.10465088,-0.4554081,-0.00762942,-0.43849918,0.3452719,0.16575441,-0.07237531,-0.2330123,-0.065847,-0.0136205,0.1953744,-0.46645334,-0.22104503,0.6245895,-0.4137238,-0.2211677,-0.7492743,-0.24493583,-0.22792265,-0.09571904,-0.5684932,0.65784484,0.5754176,0.09250989,-0.33752564,-0.4046968,0.13190551,-0.10229407,-0.7479904,-0.58035815,-0.54242194,-0.02858889,-0.52236897,0.13735509,0.03190028,0.46144024,0.21612705,-0.18649353,-0.22758846,-0.5386001,-0.2732696,0.3732061,-0.20754266,0.4834205,0.3941614,-0.47034815,-0.09126845,0.4650279,0.01436993,-0.3415681,0.2569934,0.5005066,-0.04497861,-0.6084479,-0.56855106,0.8760026,-0.03506972,-0.09533521,0.4191326,0.14930978,-0.08888599,-0.04394069,-0.04039623,-0.36642826,0.58216995,-0.1345726,0.5930351,0.17365435,-0.06952342,0.74921876,0.1231922,-0.12312243,-0.08935846,0.5296234,-0.27688262,1.2451956,-0.6753556,0.13103928,0.28871378,-0.070024,-0.97066605,0.32316265,-0.1748006,-0.03725329,0.30632395,-1.1609821,-0.1281029,0.12729521,0.2835707,-0.05662416,-0.3324342,-0.42860398,0.2698053,-0.2528898,-0.40840554,0.44478542,-0.30410245,-0.39880428,0.6651935,-1.9967008],[0.2133988,0.20425248,-0.4535314,0.1526525,-0.6483677,-0.1538792,-0.05699473,0.11061385,-0.40657225,0.14549625,-0.07254109,-0.27261925,0.30446625,0.27411976,-0.18508558,0.71853566,-0.16296647,0.33561963,-0.23153189,-0.10789214,-0.31263775,0.02776072,0.31537268,-0.27034977,0.24637668,-0.1791698,-0.18406034,0.23939057,-0.7056465,-0.3635411,-0.40894613,0.18801396,-0.31532577,0.02221796,-0.10747092,-0.1407994,0.34255537,-0.5134705,-0.25950393,-0.10634986,0.09703095,-0.22181682,-0.16311236,0.3610787,0.08504914,0.4528471,0.78525966,0.04820234,-0.64101267,-0.2028952,0.17328073,-0.15570161,0.428264,-0.53224576,0.0445985,-0.3068981,-0.15319012,0.07253429,0.3567401,-0.01309901,0.7236501,0.35282993,-0.79909605,0.49149007,0.1128035,-0.09871151,-0.83094597,0.17964108,-0.48643765,0.03802265,-0.47644743,0.8157318,-0.36239764,-0.33790007,1.0825907,0.38816842,-0.5268343,0.64067197,0.6120723,0.23352651,-0.04742884,-0.30275527,0.06902377,-0.02073595,-0.3409486,-0.10562364,-0.4556971,-0.18524906,-0.09657626,0.0795603,0.07620472,0.18205708,-0.50357777,-0.02355665,0.8411652,-0.14760645,-0.9415237,-0.26226962,-0.17696048,-0.00146111,0.24480614,-0.2892752,-0.8709971,0.38497406,-0.69720775,-0.435415,-0.212966,-0.11636405,0.13890095,0.18359014,-0.26171932,-0.16421673,0.5339428,-0.34106034,0.10198189,-0.08182421,-0.7753402,0.7913255,0.13571459,0.16438332,-0.09770007,-0.36046183,-0.1698596,0.08382584,-0.5235555,0.6647187,0.6656022,-0.17455378,0.35709777,-0.05862252,0.6267512,-0.18563414,-1.4332651,0.16862848,-0.14078271,0.08854847,-0.44213578,-0.30742976,-1.0406307,-0.19079424,0.41228735,0.22393198,0.36503717,0.06487244,0.7842909,-0.39707676,-0.5026946,0.92383116,-0.4771192,-0.5634573,0.4079619,0.04568373,-0.04345068,-0.23216139,-0.03825578,0.82252336,0.13423155,-0.17123593,-0.20028949,-0.2568126,-0.2622789,-0.09371024,0.4668868,-0.57066023,-0.12730956,-0.36374074,0.8937524,0.10133576,-0.20928356,-0.20251004,0.4624126,-0.08157147,0.88699055,-0.2684393,-0.13416408,0.3402237,0.02430284,0.07909857,-0.091868,-0.7667023,-0.55132633,-0.17104593,-0.7165359,-0.5941848,-0.18378736,0.30231282,1.0213819,-0.56477153,-1.2554042,0.82810086,0.2461451,0.01704341,0.4763951,-0.07611227,0.05198862,0.23050508,-0.35726076,0.24553563,0.19217981,-0.49267107,0.3150885,-0.352354,0.47227022,-0.39153934,0.276567,-0.15335192,0.12585734,0.11895693,0.1673258,-0.00235684,-0.57841,0.04570603,0.37715334,0.846124,0.36016223,0.11891103,-0.15951489,-0.13443205,0.32158655,0.34687635,-0.01722069,0.28681928,-0.07606465,-0.27104497,-0.3234305,0.17194843,0.15836859,-0.00859551,0.49963653,-0.48846805,0.22915888,-0.10890531,-0.01826876,0.30573428,0.49114883,-0.13995975,-0.33613,-0.3247271,0.20596877,0.07086793],[0.03774989,0.39846203,-0.00884917,-0.02588619,-0.26741698,-0.19278209,-0.3210261,0.41525123,0.2574013,0.01641947,-0.7860334,0.09896924,0.3263594,-0.35305187,-0.24082531,-0.6159619,0.53813064,0.38066724,-0.8888491,0.30634943,0.516753,0.5739245,-0.1304351,0.29983103,0.12711321,0.07825863,-0.44010642,-0.19635627,0.07519988,-0.05662575,0.3480392,-0.1469583,-0.2611133,0.11606163,0.3113493,0.05750498,-0.24648656,-0.3898167,-0.64834523,-0.66655636,-0.55123186,0.0611849,-0.7342203,0.5260333,-0.05388697,0.10251644,0.51629066,0.27712843,-0.4285676,0.17728482,-0.23429571,-0.13281098,-0.24786057,-0.24531713,0.18044958,0.3699235,-0.02347222,0.04728459,0.08659758,-0.7213736,0.01009061,-0.86509293,0.46415505,0.28176203,-0.34276477,0.5414183,-0.0444512,0.0736028,0.23781075,-0.18912709,-0.1811699,-0.13123459,-0.03429019,-0.1039809,-0.04065077,-0.33304134,-0.43362325,0.57917416,0.4953541,-0.179144,0.7667462,-0.8899058,-0.15529673,-0.60117555,-0.5285416,0.692106,-0.30695572,-0.31787583,-0.50733334,0.6159708,0.1487018,-0.03906451,0.23315443,0.04407568,-0.50444776,-0.26633036,0.06910939,0.5365156,-0.11906597,0.06213951,0.13684075,-0.4005467,0.02097053,0.18277998,-0.40099865,0.01921604,-0.06429765,-0.18747914,0.81735283,-0.1980353,0.3391774,-0.3221155,0.32583687,0.4012467,0.35869572,-0.17446065,-0.2742067,0.26741982,-1.2298919,-0.04246827,-0.05935988,-0.18101496,-0.37465104,0.42751923,-0.04973924,0.65803814,0.29222098,-0.0068057,-0.11722647,-0.003714,-0.3412977,-0.0773694,0.6686293,0.06650835,0.00543327,-0.47730476,-0.25016955,-0.41095588,0.40818775,-0.03871225,-0.9145352,-0.11646106,0.88250595,-0.3826666,-1.3445284,0.34717447,-0.30977,-0.36912778,0.23488164,0.57845104,0.49856436,0.34028345,-0.7862219,0.11982342,0.7403573,0.55298555,0.92492515,0.12472522,-0.07081985,-0.04305986,-0.21058749,-0.15052976,0.38267693,-0.36043626,-0.6889743,0.1795446,-0.12231306,-0.31603804,-0.6365864,0.283422,-0.48566988,-0.12822922,0.42483154,-0.2579108,0.08408581,0.34750912,-0.27499518,-0.6780391,0.05240019,-0.1794607,-0.5160332,0.24637069,0.48828536,0.35951555,-0.55533814,-0.0333832,-0.02689519,-0.7312012,-0.11261263,0.24900083,0.02438322,-0.00983701,0.22428033,0.1419043,0.4896837,-0.20437574,0.01498223,-0.01542883,0.09683146,0.04687691,-0.19342828,0.40022057,0.7467175,-0.03605447,-0.14441702,0.01510432,0.5929911,0.33303878,0.10471727,0.3232736,0.8138358,-0.9483052,-0.18244094,-0.01655761,-0.12274777,-0.1512995,-0.4388746,-0.11949889,-1.288379,0.21586478,-0.5759897,-0.14928448,-0.11454304,0.14749387,-0.9499162,-0.57544804,0.27910796,-0.7168263,-0.5137006,-0.05715444,-0.69550943,-0.05330804,-0.6553808,-0.3279571,0.08470218,-0.2411275,-1.1204096,0.27914461,0.32138112,-0.09017931],[5.60801744e-01,1.14164278e-01,-5.24501026e-01,1.33206099e-01,-8.92566383e-01,6.31718278e-01,4.39104617e-01,-1.20924087e-02,-2.20925644e-01,-7.31097400e-01,-7.90122971e-02,1.32843041e+00,-2.63197124e-01,1.28131002e-01,1.28716439e-01,-1.01935335e-01,-5.96431829e-02,-3.16852957e-01,-1.90337569e-01,-1.71251083e-03,-5.68113923e-01,-8.10125232e-01,2.33109161e-01,-3.91815215e-01,-4.49564792e-02,4.69582081e-01,8.17897975e-01,-8.51053417e-01,-2.93894500e-01,1.89891845e-01,9.64445949e-01,-3.03308040e-01,-4.12346534e-02,4.96197969e-01,3.84686112e-01,-3.01563025e-01,-4.81661558e-01,-1.22190535e-01,-3.06619614e-01,3.01133990e-01,7.01828361e-01,7.26428255e-03,2.25575626e-01,-4.80663814e-02,2.24745497e-02,2.56081253e-01,-2.47979105e-01,-1.45605624e+00,-1.44828334e-01,-3.67150635e-01,-3.09018582e-01,-4.06330198e-01,-8.37006494e-02,-7.03850269e-01,6.52116299e-01,4.14971024e-01,3.66931587e-01,1.39153063e-01,-8.35649967e-02,9.45437774e-02,-3.03391635e-01,5.08050084e-01,4.59284902e-01,3.71343017e-01,2.01876894e-01,-5.58655441e-01,7.57418990e-01,-2.84567118e-01,-6.91546053e-02,-4.25436139e-01,-7.71147668e-01,3.41961741e-01,-2.04552218e-01,-9.54879373e-02,2.44140595e-01,-3.76031280e-01,4.74556796e-02,-5.28986081e-02,2.11060181e-01,-1.03163636e+00,-6.58034921e-01,-8.20314527e-01,1.47648007e-01,-7.86571980e-01,-9.62053359e-01,-1.23465821e-01,-2.25932434e-01,3.22242111e-01,-3.09567273e-01,5.37452102e-01,-1.40891448e-01,-4.23941880e-01,-7.23487616e-01,9.32589769e-02,-1.24569493e-03,-6.18339181e-01,-3.24670583e-01,8.18506718e-01,-3.55563730e-01,-5.88815033e-01,-3.74371819e-02,4.23509061e-01,1.00294012e-03,-5.60098052e-01,-9.09281373e-02,4.82472420e-01,1.87165350e-01,5.14254086e-02,1.03548050e-01,-1.21603012e-01,-1.37283936e-01,-5.59771597e-01,2.18871340e-01,2.59246320e-01,-3.62496942e-01,-8.46153572e-02,-5.97503543e-01,1.28317609e-01,2.77471840e-01,9.96419489e-02,-1.07435480e-01,2.87425488e-01,-1.92240611e-01,5.17131865e-01,2.73237169e-01,3.98999304e-01,-2.98944026e-01,8.34385455e-02,2.19517142e-01,-5.31362712e-01,-1.30918756e-01,1.69792742e-01,9.86682847e-02,6.88411474e-01,6.18103407e-02,-1.78227454e-01,-4.15528655e-01,-6.32509366e-02,-1.53518662e-01,-2.30124649e-02,1.02514200e-01,2.85503387e-01,1.99140698e-01,3.13716978e-01,9.18913782e-02,2.99985945e-01,1.27677575e-01,5.45538247e-01,1.31976902e-01,2.87755430e-01,2.48263493e-01,-3.37024182e-01,-4.92253639e-02,-5.44416189e-01,4.96763969e-03,-3.55279267e-01,-4.70538706e-01,7.28732273e-02,5.91814779e-02,9.06562954e-02,-1.57209247e-01,3.68429199e-02,6.02342747e-02,6.13127470e-01,3.32595617e-01,3.45271021e-01,-1.09468304e-01,-2.08784267e-01,-3.77567023e-01,-1.10314630e-01,4.27214473e-01,3.43207687e-01,-2.30343491e-01,3.71494740e-01,5.63508384e-02,2.20951322e-03,-7.69187331e-01,2.11361498e-01,5.84199838e-03,5.09304464e-01,9.31898355e-01,4.13422175e-02,3.70538265e-01,-1.37819842e-01,-2.22219959e-01,2.24592477e-01,6.18919015e-01,-1.06382561e+00,-3.50349396e-02,-4.25857157e-01,2.33500481e-01,-8.20142031e-01,5.00722468e-01,-5.53815842e-01,5.68745494e-01,-1.38968050e-01,2.66954869e-01,-3.96281421e-01,-4.46096838e-01,1.20582514e-01,-7.33511820e-02,4.02246416e-01,-1.12433322e-01,3.82589847e-01,1.13306649e-01,-1.12578475e+00,-8.22820663e-01,2.52717674e-01,-4.90436144e-03,-2.03689411e-01,3.73169422e-01,-2.63771206e-01,-2.94526783e-03,2.51292437e-01,5.04311860e-01,-3.07783484e-01,-2.70728946e-01,2.28341520e-02,1.23450434e+00,-1.76539682e-02,-4.57726456e-02,7.20805675e-02,-6.69247925e-01,-4.74862754e-01,-5.51509798e-01,2.91996658e-01,6.64542437e-01,-4.86445695e-01,-6.53966844e-01,-2.10051447e-01,2.40272731e-01,-3.80680114e-01,-1.66703060e-01,-4.70771253e-01,-7.41058290e-01,1.78231448e-01,-2.54531950e-01,4.45320129e-01,3.48197907e-01,-5.56891114e-02],[-1.26025662e-01,-2.12291963e-02,-1.23071633e-01,2.64461368e-01,-4.77806628e-02,7.89805233e-01,4.95827124e-02,-6.39853656e-01,1.96179852e-01,-1.41876832e-01,-1.38761759e-01,4.53940690e-01,-4.51259553e-01,2.09356248e-01,2.54919112e-01,1.44439474e-01,-1.39326617e-01,7.02077329e-01,-6.16990030e-01,-4.68092086e-03,-2.93076515e-01,6.60898328e-01,3.61441553e-01,-2.10789181e-02,-4.95059937e-01,-2.95330137e-01,-1.00011833e-01,-3.18258107e-01,-1.67971745e-01,-3.42311561e-01,-1.11034298e+00,-4.07009795e-02,-1.99843675e-01,-6.18325949e-01,-2.56895155e-01,-3.09333980e-01,6.89183176e-01,1.21796457e-02,-5.17659932e-02,1.63597777e-01,-4.16156024e-01,2.26940706e-01,-7.70474434e-01,-9.20333639e-02,2.99942821e-01,-7.06785992e-02,-2.97393948e-01,-2.10745651e-02,4.51593995e-01,4.45493668e-01,-7.90548846e-02,3.24706763e-01,-2.55582243e-01,2.86545873e-01,4.36816573e-01,-1.73521847e-01,4.15789843e-01,2.18432412e-01,-8.86821523e-02,-4.81012672e-01,5.11427283e-01,3.78722876e-01,1.01720355e-01,4.18250799e-01,7.90846705e-01,6.14449263e-01,-4.39801291e-02,-3.57947230e-01,2.26852745e-01,2.43822947e-01,-4.08862352e-01,-3.23492169e-01,1.60940699e-02,2.89805830e-01,-2.37764120e-01,-8.85108709e-01,-1.94497064e-01,5.23898780e-01,1.29407629e-01,-4.76398498e-01,-1.25243112e-01,2.21569851e-01,-2.73641646e-01,-6.74144179e-02,-8.06658506e-01,1.38296485e-01,6.27553523e-01,6.06977344e-01,-8.89703631e-01,1.53517976e-01,-2.86556423e-01,7.31395721e-01,-1.66665316e-01,-2.58102089e-01,2.11276501e-01,-2.95443863e-01,-9.09534514e-01,4.98185515e-01,-3.85639548e-01,-2.88233876e-01,-6.32767498e-01,8.41986462e-02,-2.65370935e-01,5.67164123e-01,1.62129700e-01,3.71137828e-01,3.78954649e-01,-2.19969302e-01,2.18250696e-02,2.32999146e-01,3.32764924e-01,-5.67070246e-01,-4.01269197e-01,-2.82902747e-01,-7.56060779e-01,1.71831734e-02,-1.90006867e-01,2.45130092e-01,-2.19026413e-02,4.08453256e-01,-4.90488321e-01,-1.47459477e-01,-6.70486450e-01,-2.91618884e-01,-4.95068669e-01,4.44174469e-01,-4.05202478e-01,4.72407818e-01,3.11196566e-01,-4.71757799e-01,1.76299497e-01,-8.89970481e-01,-1.31861106e-01,1.83894396e-01,-5.76324798e-02,-1.03054516e-01,-2.82715529e-01,1.14224225e-01,2.39198074e-01,2.05984235e-01,-1.33298552e-02,2.12528050e-01,-3.27646822e-01,-4.36244249e-01,-1.86550826e-01,-3.09334006e-02,-2.21414641e-02,-9.04822797e-02,-6.40061378e-01,-1.11569641e-02,6.71078801e-01,-6.63627028e-01,1.42820075e-01,6.65477037e-01,-7.33219413e-03,2.27959268e-02,9.54762548e-02,6.27834082e-01,-4.10885692e-01,3.74559700e-01,-1.14022508e-01,8.99864081e-03,2.00357158e-02,-6.03505135e-01,-6.13484144e-01,-4.52485383e-01,-1.87855899e-01,1.06150225e-01,-1.03424452e-01,5.48048258e-01,6.96625635e-02,1.78862542e-01,-1.45267665e-01,2.94851512e-01,-3.20208400e-01,-5.32293916e-01,-4.83940661e-01,2.22810820e-01,4.76126373e-02,-9.84219253e-01,-1.96272507e-02,-4.79251653e-01,2.05050185e-01,-2.87217289e-01,3.03885162e-01,-1.67908144e+00,5.46624251e-02,-3.18500549e-01,-3.69380444e-01,2.17435166e-01,1.49107397e-01,3.00525948e-02,1.97832137e-01,2.72456110e-01,-2.80401260e-01,-3.01324546e-01,7.14672744e-01,-2.20119506e-01,-1.68205035e-04,-7.57857621e-01,8.24511230e-01,5.50771773e-01,-8.71634901e-01,4.54953700e-01,3.17151517e-01,6.15984797e-01,-2.00198755e-01,-4.91158247e-01,4.84375387e-01,4.96163696e-01,1.07382670e-01,-5.85261583e-01,-3.63581777e-02,1.17240446e-02,1.05822670e+00,2.20677629e-01,-3.77134711e-01,3.68593991e-01,-8.20014954e-01,-5.25013328e-01,6.65250793e-02,6.11361444e-01,-5.89086294e-01,6.42736316e-01,-7.49425173e-01,-9.18427706e-02,-4.38709706e-01,5.72239906e-02,-4.23033208e-01,1.07275844e-02,-2.13857532e-01,7.14929998e-01,-1.12728810e+00,3.87553751e-01,1.66591302e-01,-1.75744191e-01,-2.96881557e-01,-7.29839742e-01,-2.79545873e-01,-9.75966640e-03],[2.48636365e-01,-1.89469069e-01,-5.24847150e-01,-1.14901468e-01,-1.24425255e-01,6.69355333e-01,-4.76891249e-01,1.88759297e-01,-2.51208276e-01,3.14185530e-01,-5.50820410e-01,8.41843933e-02,8.52731168e-02,8.44226331e-02,-6.33117184e-02,-1.12129629e-01,2.17048489e-02,-7.20157504e-01,-4.69891243e-02,-1.69081002e-01,-4.38376009e-01,6.86678350e-01,1.74480751e-01,-9.07680809e-01,-5.73856421e-02,2.31759548e-01,6.04406118e-01,-2.95787483e-01,6.38933405e-02,7.28406906e-01,-2.86165267e-01,-1.26614556e-01,-2.46294841e-01,-3.83598208e-01,1.37309909e-01,-2.65666693e-01,-3.03549200e-01,2.75649112e-02,-3.21436644e-01,5.09025812e-01,1.39687762e-01,1.47788331e-01,3.64855617e-01,4.00264263e-01,-4.18668315e-02,3.99800211e-01,-1.01862919e+00,1.36841655e-01,-3.10634732e-01,3.37450087e-01,7.29199615e-04,-1.63390779e+00,8.26536044e-02,3.51470441e-01,-7.82006457e-02,-4.30984825e-01,-1.03447378e+00,-5.38618505e-01,-2.33206272e-01,9.82002258e-01,1.18329190e-01,-4.09268945e-01,-6.37896478e-01,-1.41304895e-01,-2.36999691e-01,2.04850703e-01,-2.75111616e-01,-4.21834171e-01,-3.08706820e-01,-6.54436881e-03,3.85580033e-01,5.92935622e-01,1.11558251e-01,1.08315714e-01,-1.73683643e-01,3.27809423e-01,-3.43248188e-01,5.83047092e-01,6.32422447e-01,2.56043315e-01,-9.31077003e-02,-4.18148994e-01,-3.47060293e-01,-6.63228393e-01,-5.93467988e-02,3.56923006e-02,2.32581645e-01,2.09089845e-01,3.77787650e-01,5.00506870e-02,-7.74228990e-01,-3.24030280e-01,1.75947189e-01,-2.93348610e-01,-6.77385092e-01,4.67202038e-01,1.22271992e-01,3.87341172e-01,-4.81198251e-01,2.50805020e-01,3.35486770e-01,-3.36499155e-01,-3.34230572e-01,4.39678997e-01,-7.84678400e-01,4.52198312e-02,3.74275178e-01,-2.81416476e-01,4.02524263e-01,1.40250757e-01,-6.61687553e-01,4.44639295e-01,-7.37249732e-01,4.07070488e-01,-2.90331632e-01,-1.28772199e-01,4.37249780e-01,-2.96196848e-01,-5.69751263e-01,2.39967555e-01,-2.52709180e-01,-6.66180074e-01,-7.82069191e-02,2.97977924e-01,-6.61206603e-01,1.71235818e-02,1.97168395e-01,-9.01341438e-01,3.53557877e-02,-8.38319242e-01,-3.15037996e-01,-1.24729179e-01,-4.05235291e-01,9.55950022e-01,5.89141190e-01,-2.59101808e-01,-4.92578328e-01,-7.82543838e-01,-2.18688309e-01,1.10986736e-03,-8.68892610e-01,-1.02882349e+00,-1.60147950e-01,1.65568627e-02,5.29401183e-01,4.51568335e-01,-8.25545192e-01,-5.72106421e-01,4.49506491e-01,-1.67294073e+00,5.46119034e-01,5.54881155e-01,5.39742410e-01,-3.38900805e-01,8.04787129e-02,4.53666359e-01,1.46272361e-01,-3.87465239e-01,1.89269587e-01,-3.99038941e-01,-3.44812274e-01,7.87658632e-01,-2.40110084e-01,8.53421316e-02,-9.47708011e-01,-3.04217041e-01,5.02929389e-02,4.47943270e-01,-1.00554991e+00,-6.17254376e-01,-7.75767267e-01,1.86840892e-01,2.52942920e-01,-2.71698058e-01,-6.58094808e-02,-2.18046367e-01,-6.85966074e-01,-6.95392370e-01,-2.06422672e-01,-5.12711741e-02,2.56735772e-01,-8.03765178e-01,9.39512178e-02,-7.38851786e-01,-2.69393206e-01,9.35293883e-02,-2.11811781e-01,-1.77502260e-01,-9.31695044e-01,-5.23681223e-01,-2.03722939e-01,2.18889788e-01,-3.81874412e-01,2.15478927e-01,5.32144345e-02,-1.66448399e-01,-5.32550633e-01,-9.20237958e-01,-6.55607879e-01,3.10746916e-02,-3.28605711e-01,2.69714892e-01,1.58739448e-01,3.71703118e-01,2.18649134e-01,-7.89295852e-01,-7.30386317e-01,6.35679245e-01,-7.40299702e-01,5.09313166e-01,-1.37444183e-01,-4.37329292e-01,3.37621212e-01,-1.72974035e-01,-1.23684324e-01,-1.06013849e-01,1.05808347e-01,-3.20154697e-01,-2.55574584e-01,-4.02395129e-01,4.49188858e-01,6.95696115e-01,1.87436447e-01,-6.38521984e-02,-2.53231853e-01,1.70613870e-01,8.04161951e-02,-1.33253828e-01,-4.41015601e-01,-1.23597488e-01,4.61758971e-02,3.38666588e-02,1.59520119e-01,2.68278629e-01,-5.91519535e-01,-7.05467939e-01,-2.85978913e-01,1.52768403e-01,1.38572246e-01,2.14632764e-01],[-0.23575313,-0.23098874,0.45741853,-0.11670028,-0.624877,0.39043856,-0.0023291,-0.44852367,-0.04617114,-0.87124413,0.12467166,-0.35637686,-0.00548793,-0.25314796,-0.25216657,0.28556657,0.17178468,0.47949424,-0.24638672,0.09596167,0.52501583,-0.08438729,0.45324245,0.68779325,-0.22243364,-0.23043407,0.1853628,0.13561942,0.13190365,0.4696445,-0.6672364,-0.53191286,-0.32960036,0.01943845,-0.2270297,-0.13042092,-0.27718475,-0.4421525,0.06834214,-0.24658413,-0.5380617,0.21294972,0.7091936,0.23425727,-0.12646823,-0.04600772,0.05048348,0.36339286,-0.5233239,-0.08218817,-0.09206513,0.14700389,0.59334534,0.44364265,-0.15671623,-0.05123618,0.07712181,0.18995352,-0.07251853,0.19415541,0.25054777,0.28129026,-0.30284828,0.28698713,0.29707488,-0.5812965,0.16456151,0.48317707,-0.3100785,0.41305023,0.24181013,-0.29438958,0.20324561,0.49140528,-0.20308168,0.55526817,0.06569669,0.43566787,0.17683828,0.4235482,-0.15840231,-0.54893285,-0.29648894,0.36896357,-0.37662682,0.06548522,-0.05638904,0.12348229,-0.5760875,0.04093214,-0.1899794,0.3540617,-0.38799006,0.35506162,-0.85289127,0.02542284,-0.12059637,0.34905624,-0.22524454,0.20201957,-0.03648375,0.22287177,-0.28439185,0.33100143,-0.04940891,-0.45749465,0.13546695,0.22729313,0.01116889,0.30204633,-0.16223657,-0.14591718,0.22657022,-0.06726411,-0.3850017,-0.30544877,-0.4949442,-0.5297737,0.22238496,0.8867526,-0.1736751,0.16622333,0.43708494,0.10473108,-0.00749808,-0.6707366,0.43850228,0.5926661,0.28472608,-0.45370743,-0.19925718,-0.54242617,-0.12411568,0.33601588,0.3269688,0.04939546,-0.32266405,0.1186,-0.1595176,-0.4054893,-0.01534705,-0.0376295,0.0576882,-0.7831715,-0.14639607,0.26580748,0.22645001,0.5612112,0.51587045,0.03359463,0.07499496,-0.06090324,-0.18035787,0.4121058,-0.20570739,-0.05262166,-0.05886372,0.72688556,-0.19562542,0.6413811,0.08971873,0.26973417,0.02334809,-0.5472214,0.05345724,-0.2919746,0.08782747,0.3590983,-0.81448233,0.3811176,0.11482482,0.07062397,-0.55664915,0.39515358,0.00796079,0.17639881,0.46604028,-0.13946357,-0.46345297,0.02026312,-0.3547191,0.03360952,0.07630748,-0.10105732,-0.20910655,-0.5433117,0.0448346,0.5494937,0.45363244,1.6217033,-0.28971046,-0.23598064,0.44932824,-0.2098921,-0.20276698,-0.70977277,-0.32601473,-0.2066169,-0.6415112,0.6005001,0.10357058,-0.08001652,0.10222241,0.4020824,-0.7139818,0.63085884,0.26521757,-0.15683275,-0.16458587,-0.47030064,-0.45620444,0.05113992,-0.57877433,-0.60283524,0.22541104,-0.07243852,-0.01511037,0.20802031,0.02609682,-0.02583013,-0.14536402,-0.12001608,0.29996306,0.39047292,0.17207421,-0.7110893,0.3299176,0.00852821,0.01462025,-0.3387761,-0.30472264,-0.10746783,-0.03989404,0.05196574,-0.02978279,0.06712985,-0.28825736,-0.31728485,0.26382187,0.4057986],[3.16092461e-01,1.90513313e-01,-2.93652743e-01,-4.00068194e-01,-3.07611465e-01,-8.02054480e-02,3.38417739e-01,2.24267736e-01,-1.32680712e-02,-7.75640965e-01,-2.95814246e-01,1.05692275e-01,2.97524452e-01,-3.42368394e-01,-4.46346104e-01,2.57595628e-01,-1.01258367e-01,-7.39358738e-02,-6.46387815e-01,-7.21946657e-01,5.12630463e-01,-1.13231510e-01,5.23391902e-01,5.20592868e-01,2.14443401e-01,-1.87021032e-01,-8.93541351e-02,-1.99388370e-01,1.72735363e-01,1.24260753e-01,7.71633506e-01,6.07811451e-01,-1.37898073e-01,-5.42494118e-01,1.23612788e-02,4.73342370e-03,-6.27631098e-02,-3.70237678e-01,3.71703446e-01,-1.77667156e-01,-5.16124845e-01,1.69851437e-01,-5.55025160e-01,-5.18221855e-01,2.27121785e-01,6.03094518e-01,-5.51762104e-01,-6.08460344e-02,4.72276241e-01,-5.54538727e-01,1.71356052e-02,5.36589444e-01,-2.76560605e-01,1.21117558e-03,-4.39220108e-04,7.57086352e-02,5.17713368e-01,3.42444070e-02,7.15311691e-02,3.11790556e-01,-3.07834357e-01,7.74006546e-01,-6.40187860e-01,6.84840754e-02,4.53153342e-01,-8.51489604e-02,-8.96445155e-01,-1.74823776e-01,-2.19257653e-01,-2.10557118e-01,-1.91120803e-01,3.28431249e-01,-7.17137992e-01,3.51296455e-01,6.24510467e-01,6.20200634e-01,1.60321519e-01,-7.74152517e-01,4.19516742e-01,2.17561379e-01,-5.66510022e-01,4.65317279e-01,2.70300340e-02,-2.68077642e-01,-8.80580768e-02,1.99834049e-01,-2.07191855e-02,2.36764401e-01,-9.10521686e-01,-7.28423595e-02,-3.27730745e-01,-1.91643745e-01,-1.36129856e-01,4.30345953e-01,-8.82662535e-02,-5.99079370e-01,-2.33336203e-02,-2.68408269e-01,4.38646615e-01,5.26901543e-01,1.71932518e-01,-9.06637132e-01,-7.71588624e-01,3.29982400e-01,1.74678475e-01,1.77974403e-01,-4.39332575e-01,-5.74935436e-01,-4.63499486e-01,-2.54350871e-01,4.74557802e-02,3.46299767e-01,5.59578717e-01,3.64444017e-01,1.66103020e-01,-2.88421899e-01,-1.13714063e+00,-6.92232966e-01,4.46610689e-01,3.69168460e-01,3.80934000e-01,2.26286426e-01,-7.48334289e-01,-4.78221208e-01,-3.37809056e-01,5.50468624e-01,-4.91519958e-01,2.01545596e-01,-9.06692088e-01,2.36200601e-01,-1.28133357e-01,-9.18159448e-03,-2.80793607e-01,-3.58220190e-01,4.92511094e-01,-2.52837509e-01,-1.23566556e+00,2.28914127e-01,2.92153388e-01,-1.18824339e+00,-1.14201508e-01,1.01013720e-01,-6.87758178e-02,1.15487717e-01,-2.34848261e-02,-1.78879249e+00,-2.90973753e-01,4.19355243e-01,7.20340163e-02,-5.68826377e-01,-1.06002754e-02,5.06320119e-01,-4.26547945e-01,1.84058234e-01,-4.46266353e-01,-3.40913296e-01,2.17202276e-01,-4.42069054e-01,-2.48886362e-01,-6.24380112e-01,-3.62023026e-01,2.91175932e-01,-4.48828749e-02,2.01872215e-01,1.02955826e-01,-5.60234904e-01,-1.63343683e-01,7.42219746e-01,3.10700893e-01,-1.49264205e+00,7.87421703e-01,-2.96682626e-01,5.38527071e-01,5.85035920e-01,1.37423828e-01,3.55965793e-01,6.93193257e-01,2.43273690e-01,-1.04684616e-02,2.18008637e-01,2.70062655e-01,-4.37920600e-01,-9.43299606e-02,-2.20780075e-01,-2.17354774e-01,5.20215556e-02,-1.06541142e-01,4.39305678e-02,-7.91372240e-01,2.38691583e-01,-1.73817590e-01,-4.18408066e-01,-1.03214860e-01,7.40342081e-01,1.76406533e-01,2.41514266e-01,3.81031513e-01,-6.73391640e-01,-2.95519292e-01,-2.27646992e-01,2.75722027e-01,4.58258629e-01,-4.67394665e-02,-4.66041267e-01,1.42441392e-01,-1.53735384e-01,1.67331353e-01,-1.00459874e+00,-6.78018630e-01,-9.32463586e-01,1.17391057e-01,2.02506125e-01,2.36155286e-01,2.27967888e-01,-4.67037149e-02,2.16114208e-01,-1.07463694e+00,2.37608269e-01,-8.75476748e-02,1.93897076e-02,-4.29921329e-01,2.35999882e-01,-1.20348498e-01,-1.94954291e-01,-9.97014493e-02,5.62002063e-01,1.38764083e-01,2.61172950e-01,4.21126187e-01,2.68182486e-01,3.15312922e-01,-1.52259290e+00,3.50148320e-01,-4.21369463e-01,6.30791187e-01,2.30415583e-01,-2.69135416e-01,2.29558215e-01,-2.07024366e-01,2.76114736e-02],[-6.67039454e-01,-1.65240437e-01,-4.29025799e-01,-1.54131815e-01,4.15246189e-02,-2.25667968e-01,-8.17315355e-02,-8.91277671e-01,3.63069117e-01,-6.51854992e-01,-6.50186150e-04,-4.86216456e-01,6.19682558e-02,6.05932832e-01,-3.98770839e-01,1.01783916e-01,-2.77270633e-03,1.63759924e-02,-1.36844248e-01,5.04422247e-01,-5.96624136e-01,2.05286980e-01,-1.44814163e-01,-1.82458848e-01,3.90332788e-01,-5.04455388e-01,5.66170812e-01,-2.82474786e-01,-2.91475266e-01,1.05098158e-01,-4.29855287e-02,1.06050467e+00,-8.43106329e-01,-1.23272203e-01,1.67301614e-02,-3.75484735e-01,-3.46676201e-01,3.86811137e-01,3.64556402e-01,1.71903729e-01,-3.21003273e-02,-6.14627719e-01,6.40700877e-01,-5.09682834e-01,4.39783007e-01,1.32271409e-01,-7.12260246e-01,3.08749259e-01,-3.23729068e-01,-4.39179450e-01,-2.47020185e-01,3.83993208e-01,7.68370688e-01,-1.35950267e-01,2.35878676e-03,3.56851071e-01,-2.97030360e-01,3.00665706e-01,3.03635180e-01,-5.69871962e-01,3.87896508e-01,1.83877181e-02,6.38246298e-01,-1.48099974e-01,-3.49606156e-01,4.46222723e-01,1.37696611e-02,-6.13318980e-01,1.39623761e-01,7.88496248e-03,-1.26646727e-01,-4.38252658e-01,-6.00219846e-01,2.94228375e-01,4.24940437e-01,9.26652923e-02,-1.68422401e-01,-3.26255292e-01,-4.15416181e-01,8.19943398e-02,-3.39718163e-01,4.13747817e-01,-2.39176035e-01,-7.50673041e-02,-7.53397271e-02,-4.14797634e-01,3.05138916e-01,-1.70923784e-01,2.07364857e-02,1.71369061e-01,-1.41906589e-01,4.06747431e-01,7.47630745e-02,-1.03460813e+00,-5.74058890e-01,1.28419876e-01,-6.03176415e-01,-1.58584967e-01,-4.46790487e-01,-4.32207197e-01,-3.71925123e-02,-2.32248947e-01,4.89333510e-01,-1.64683685e-02,2.54081577e-01,-4.80561912e-01,-2.66771555e-01,5.33358216e-01,7.67467082e-01,-1.20916916e-03,-2.91984946e-01,-4.78219837e-01,5.96367009e-03,-1.87647030e-01,2.64966458e-01,2.91842252e-01,5.68459868e-01,-1.52888568e-03,-4.40943480e-01,3.38755772e-02,-6.64409995e-01,6.60565138e-01,-9.06822503e-01,-3.76116782e-01,-1.03276595e-01,-1.81178167e-01,1.86971977e-01,1.14112280e-01,-7.45113418e-02,-5.28451324e-01,1.07492298e-01,3.80274594e-01,-6.63404703e-01,-3.32706988e-01,6.09607622e-02,-5.75119436e-01,-5.59548318e-01,3.27158481e-01,8.63651037e-02,-3.64409178e-01,-2.31532842e-01,-1.43246025e-01,2.27737892e-02,7.91750103e-02,2.53799230e-01,4.46146578e-01,-2.08419085e-01,5.98005772e-01,3.09853047e-01,-2.84626603e-01,4.31497931e-01,7.97894597e-02,4.72974718e-01,-5.02211094e-01,1.37145475e-01,-1.37026235e-02,-5.55644572e-01,-3.77554029e-01,-6.19375527e-01,5.41710138e-01,2.22366005e-01,-7.50781894e-01,-5.52781522e-01,-2.87177324e-01,-5.87261856e-01,9.32230130e-02,-5.55511832e-01,-3.84853989e-01,-1.19681591e-02,8.33687782e-01,2.84903347e-01,-4.58985800e-03,-4.13882464e-01,1.21860057e-01,6.22049682e-02,-9.02220905e-01,-2.03527883e-01,-3.90672177e-01,-3.43421757e-01,1.40847147e-01,-1.78349063e-01,3.00534725e-01,-3.33424926e-01,-7.32967794e-01,-4.64885324e-01,2.08462656e-01,-1.22161046e-01,-1.05774415e+00,2.67977625e-01,-1.11054882e-01,2.90977299e-01,3.47478777e-01,-5.06122530e-01,-3.10620755e-01,9.32953954e-02,-3.89050752e-01,-1.26557827e-01,1.15871839e-01,-2.59973466e-01,-1.14299141e-01,3.82953346e-01,5.49972467e-02,2.57092476e-01,1.34375542e-02,7.04839289e-01,-1.43986091e-01,-1.00163102e+00,-1.02847564e+00,-5.27162313e-01,6.91950992e-02,-8.15798700e-01,2.28606269e-01,4.35601532e-01,4.32033271e-01,-6.31506205e-01,6.33942068e-01,3.64080817e-01,6.37893856e-01,-5.42348474e-02,5.97656906e-01,3.23291481e-01,6.49241686e-01,6.87495112e-01,-5.36256373e-01,-4.74293321e-01,-4.47013944e-01,5.53723335e-01,-2.71763783e-02,-4.03928369e-01,-4.19314891e-01,9.05487239e-02,1.93692669e-02,6.16780877e-01,-4.54490215e-01,-8.67994130e-02,-3.68931741e-01,-3.29018623e-01,2.28378430e-01,2.35123280e-02,4.05188322e-01],[-7.90086508e-01,-1.70132071e-01,-9.39234197e-01,-2.25007743e-01,-6.51165783e-01,6.73849642e-01,-4.74327475e-01,-1.33680748e-02,1.10578053e-01,2.55015552e-01,-5.74921966e-01,5.28265655e-01,-1.51292980e-01,1.88250765e-01,2.54591256e-01,-8.43092725e-02,-3.01189512e-01,-7.81829357e-02,3.40029210e-01,2.02266902e-01,-9.72318113e-01,7.47141838e-01,7.13231787e-02,1.37036943e+00,1.23130800e-02,-5.16900659e-01,1.06408097e-01,-3.92344445e-02,-4.48891195e-03,-2.03360394e-01,-5.42753577e-01,-2.45741472e-01,1.57744244e-01,-6.10954352e-02,2.79570192e-01,-3.43595207e-01,-4.09450024e-01,3.16930681e-01,-2.46775120e-01,2.41867099e-02,2.43709564e-01,-5.53064585e-01,4.86217253e-02,-1.13461971e-01,-3.09674814e-02,-1.11269772e+00,3.88963759e-01,3.71332705e-01,5.42196929e-01,-1.80997789e-01,1.32572457e-01,-7.72114336e-01,2.80256122e-01,-7.28344798e-01,-4.29251641e-01,5.89809060e-01,-4.62918311e-01,-9.33615923e-01,-5.31434655e-01,1.88158546e-02,8.95225853e-02,-3.65008205e-01,5.06815732e-01,1.09303638e-01,-2.00405627e-01,1.33702025e-01,-2.55841706e-02,-4.49991107e-01,-3.57472748e-01,1.53434947e-02,-2.56047845e-01,3.37595046e-01,-9.25823450e-02,-5.86795509e-01,-1.91908658e-01,1.02008931e-01,3.19756806e-01,-6.12331331e-01,-3.85065764e-01,6.16636500e-02,-5.13414979e-01,8.08170885e-02,3.58996600e-01,3.47823113e-01,7.15249330e-02,3.96957062e-02,2.56196737e-01,6.52172327e-01,1.70734167e-01,2.44503960e-01,-6.33712769e-01,9.15552899e-02,-4.78730828e-01,7.59875923e-02,-1.63552105e-01,5.25230467e-01,-1.66167170e-01,4.26854521e-01,-3.99922490e-01,-3.47380817e-01,-7.22065151e-01,-1.12820700e-01,-3.15352648e-01,-1.73089355e-01,-7.47969747e-01,-1.99247167e-01,-1.67529374e-01,-4.09924835e-01,1.04312122e-01,-3.02428007e-01,4.24964845e-01,4.15578395e-01,-6.17948413e-01,5.04375577e-01,7.28642419e-02,-2.00149834e-01,7.93062806e-01,-8.98710370e-01,4.48840670e-02,1.18733488e-01,-1.55269548e-01,-3.53072971e-01,2.30447754e-01,9.11113620e-02,-5.73771358e-01,3.96388769e-01,2.12528914e-01,5.10995351e-02,4.08772975e-01,-2.56018490e-01,1.09196216e-01,4.57042038e-01,9.54863802e-02,-1.07423520e+00,8.99567008e-02,-5.42989731e-01,3.51592869e-01,-6.51377261e-01,-4.90980357e-01,1.20390795e-01,-5.57956994e-01,5.98159023e-02,2.59391457e-01,-1.66919068e-01,3.96060467e-01,-7.42254555e-01,-5.53878546e-01,5.40285110e-01,8.08862150e-01,-3.74017984e-01,8.49524438e-01,1.46407247e-01,4.29531455e-01,-1.51275381e-01,1.29130408e-01,3.68720353e-01,-2.51052022e-01,-3.89346987e-01,-2.83082183e-02,4.38586712e-01,1.29447610e-03,4.62887466e-01,1.24586649e-01,-1.04549515e+00,3.75824481e-01,-6.96687400e-01,2.66310543e-01,-1.97546612e-02,-3.05734128e-01,-1.11481138e-01,6.87160194e-02,-5.13572581e-02,4.35753107e-01,-7.51230836e-01,-4.61825579e-01,-1.49745613e-01,-1.48442507e+00,-5.98632991e-01,3.24052662e-01,1.68071523e-01,-1.47072487e-02,-5.15370846e-01,3.93455684e-01,-4.96641882e-02,-3.21445137e-01,5.23895025e-01,-5.51754653e-01,-8.84704366e-02,2.05686778e-01,-9.90391195e-01,7.90187567e-02,3.78344655e-02,9.33679342e-02,-2.05635160e-01,-2.59249330e-01,6.44985959e-02,-2.62932796e-02,-8.72280538e-01,1.15069307e-01,-5.51481485e-01,-2.17686780e-02,-1.38246089e-01,4.50524122e-01,-5.17958045e-01,-2.65007734e-01,4.20455774e-03,1.71371341e-01,-8.77141237e-01,4.46520090e-01,-1.63158357e-01,-2.28186682e-01,-6.48618937e-02,3.16707790e-01,9.64900330e-02,3.10435385e-01,-7.16134071e-01,-2.33661190e-01,5.44941843e-01,5.43429703e-02,4.18238789e-01,-1.26963869e-01,-1.03305101e-01,-2.20582038e-01,4.43810910e-01,2.09176511e-01,-8.68068814e-01,6.63204670e-01,1.05945423e-01,-1.49678022e-01,8.17585140e-02,-1.75877213e-01,8.74985218e-01,-1.31740585e-01,-6.67692363e-01,-2.74990380e-01,-1.39518939e-02,2.54811406e-01,7.14624003e-02,5.68439901e-01,4.68786031e-01],[9.87783521e-02,1.00935948e+00,-5.79506814e-01,-8.40879202e-01,-3.16396683e-01,-1.15336108e+00,-8.45799387e-01,-5.20774364e-01,7.86704302e-01,-3.66334826e-01,2.85555601e-01,-3.17582525e-02,6.24352843e-02,-3.42575222e-01,-7.81665623e-01,4.47864681e-01,-1.26361877e-01,-2.94493288e-01,-1.70919269e-01,-4.73547578e-01,-3.20119530e-01,2.26070657e-01,6.05603635e-01,9.94139135e-01,-3.64659607e-01,-2.23262444e-01,3.86386700e-02,2.43873700e-01,-1.18612123e+00,6.22634351e-01,4.55372810e-01,-5.34080923e-01,-3.94485354e-01,-2.34908924e-01,7.15596437e-01,7.14868426e-01,-6.19443536e-01,1.32984817e-01,2.22860828e-01,6.35399580e-01,-2.55737990e-01,-2.69129962e-01,-2.44666710e-02,-5.19144058e-01,9.36088264e-01,5.65673150e-02,-2.40479156e-01,9.89315808e-02,-2.51613408e-01,1.98784754e-01,-6.92627311e-01,-8.21100831e-01,-5.18801987e-01,3.32004242e-02,4.86211121e-01,2.70229906e-01,1.72446549e-01,5.74686050e-01,3.75927657e-01,1.13897848e+00,3.75659198e-01,2.96600997e-01,-4.71765071e-01,-2.32017979e-01,9.07556787e-02,-7.83027112e-02,1.78381488e-01,-1.68587908e-01,9.64424312e-02,4.43531536e-02,3.67345244e-01,7.62180537e-02,4.05374289e-01,4.23579067e-01,-2.50337929e-01,3.57038766e-01,3.24656591e-02,-9.21402425e-02,-4.57200080e-01,-4.23400909e-01,-1.65584877e-01,2.15106919e-01,2.36721486e-01,-2.04995409e-01,3.77818793e-01,-1.67240128e-01,-2.03728124e-01,-1.77122548e-01,-7.90642262e-01,2.97003500e-02,-1.78881455e-02,4.92181420e-01,-7.41785526e-01,-6.53527081e-02,3.86129826e-01,3.58885080e-02,-2.28001446e-01,7.84994587e-02,2.17948437e-01,-8.37810814e-01,-6.95837259e-01,-9.09832418e-01,-1.29699320e-01,-5.66235222e-02,-8.96362782e-01,-2.23576292e-01,5.07080674e-01,3.59300077e-01,-1.06090212e+00,-5.89688241e-01,-2.08334908e-01,2.48061568e-02,8.99762958e-02,-7.56878108e-02,7.03878164e-01,-1.01235248e-02,3.09194513e-02,2.27973104e-01,-5.62840402e-01,6.01652861e-01,8.65522474e-02,1.73376054e-01,-8.10370147e-01,-5.76236010e-01,4.76825982e-01,1.13583632e-01,-4.91181575e-02,-3.03469270e-01,-3.79551739e-01,-4.12548155e-01,-6.35678589e-01,-6.57990992e-01,-1.76337457e+00,-5.93094945e-01,4.56980541e-02,1.77350640e-01,8.04782733e-02,-8.34327221e-01,-1.07140794e-01,-6.12756729e-01,-1.11160584e-01,4.49241400e-01,-2.42071584e-01,-8.69824588e-02,-7.59735107e-01,-1.71181262e-01,-8.21184695e-01,-2.54085865e-02,-2.04870462e-01,-5.58389127e-02,-1.01780005e-01,-2.78531432e-01,6.14912510e-01,-5.38840175e-01,-8.83879364e-01,6.04532510e-02,-9.41406429e-01,1.48845196e-01,-6.93886697e-01,1.68931577e-02,1.01402664e+00,2.71830410e-01,8.06462020e-02,-6.43991649e-01,-2.40338445e-01,-5.21064281e-01,-1.24453738e-01,3.44327301e-01,-8.70265603e-01,-1.60340786e+00,-3.37674797e-01,-8.23595285e-01,1.71250433e-01,-1.35843739e-01,-2.02187106e-01,-7.13232636e-01,1.51277587e-01,-1.18392922e-01,1.33436704e-02,-2.90355057e-01,-1.50058810e-02,7.43355334e-01,6.88438952e-01,6.28327310e-01,-2.65006453e-01,-3.92641157e-01,-4.14770573e-01,-4.71404076e-01,3.04882139e-01,4.54820283e-02,-2.53131002e-01,-6.35776937e-01,-2.96587437e-01,4.52371329e-01,-2.02593699e-01,-5.08307636e-01,-6.94822073e-01,-3.38199437e-01,-2.22107366e-01,-4.18393731e-01,-2.61274397e-01,2.30895773e-01,2.66327653e-02,1.45420089e-01,-6.82082772e-02,1.33947581e-02,2.05519218e-02,4.79899883e-01,-8.89373243e-01,2.02142864e-01,2.39254739e-02,-2.68778443e-01,2.51720876e-01,-9.85838026e-02,3.35188746e-01,-2.60121495e-01,3.45600516e-01,2.84930825e-01,-4.03017223e-01,-3.10646802e-01,4.49852198e-01,7.65162408e-01,5.17728508e-01,-7.91295171e-02,1.73718601e-01,-1.97335482e-01,-2.64887780e-01,-1.50773391e-01,6.47064894e-02,-1.19787574e-01,1.23989955e-03,7.08104745e-02,8.81795958e-02,2.48435512e-01,-6.31830618e-02,-8.68437767e-01,8.70679557e-01,-2.09763239e-04,-1.68600941e+00,-7.29009688e-01],[-6.42765686e-02,-7.29737520e-01,4.21798348e-01,-6.35751188e-01,1.45055503e-01,4.69743967e-01,2.12713897e-01,-4.74992841e-02,-9.60638463e-01,-3.70082289e-01,-4.24942493e-01,-3.32136303e-01,-2.69441810e-02,-1.10745680e+00,-9.18433368e-02,-3.49930316e-01,9.97837484e-02,-1.85945347e-01,-1.18771352e-01,1.00721411e-01,-3.76267970e-01,-1.20820627e-01,1.29897550e-01,3.21614206e-01,3.42063546e-01,3.42638105e-01,7.15394080e-01,9.39408839e-01,1.34852817e-02,-1.16851255e-01,-3.21129650e-01,3.09975147e-01,-1.44368875e+00,7.57974312e-02,6.95203394e-02,1.22577541e-01,-1.65320724e-01,8.47645760e-01,2.42888909e-02,2.35184580e-02,-3.16368528e-02,-5.36773980e-01,-2.99577112e-03,1.10714033e-01,-3.18232834e-01,2.39380836e-01,1.91245019e-01,1.83935463e-01,2.99713463e-01,-5.43162674e-02,-5.06326556e-02,6.69032574e-01,2.13145360e-01,1.51119292e-01,-6.67816579e-01,2.75815010e-01,4.65695739e-01,-1.36660784e-01,-9.57476273e-02,1.98295131e-01,-2.18227506e-01,2.39336789e-02,-3.98375541e-01,-7.38780081e-01,-7.31630683e-01,1.00910924e-02,2.00548619e-01,-1.07900098e-01,1.06389960e-02,6.03868723e-01,3.10435504e-01,2.97242522e-01,3.81057680e-01,4.61593358e-04,1.10509157e+00,3.41461182e-01,6.16747178e-02,2.65286416e-01,3.59542102e-01,-6.17508553e-02,-5.23413062e-01,4.61984217e-01,-7.98767686e-01,-3.91815573e-01,-6.00537300e-01,2.41827399e-01,6.20983303e-01,-2.45778084e-01,2.81867236e-01,-4.21989918e-01,-5.48572481e-01,3.43756258e-01,-6.07352220e-02,7.33306885e-01,2.86318451e-01,1.72658548e-01,5.26719868e-01,3.73408198e-01,-7.39306509e-01,-7.51437724e-01,-1.49821788e-01,-5.75055003e-01,2.79002190e-01,-1.96938381e-01,7.92288303e-01,3.55977207e-01,-7.74961635e-02,-3.15411061e-01,2.75386661e-01,-3.82122844e-01,-3.98003876e-01,8.37978795e-02,-5.02634287e-01,-3.13787222e-01,6.82383403e-02,-6.67482018e-01,-3.56187940e-01,6.55909836e-01,4.19342816e-02,-1.86345860e-01,-1.57250643e-01,4.66221005e-01,1.34438306e-01,-1.97969735e-01,1.39113264e-02,-3.87387484e-01,4.12762105e-01,9.22115445e-02,3.54523480e-01,-3.01876575e-01,-4.15462613e-01,2.46643201e-01,5.64016640e-01,-4.54497099e-01,6.25604391e-01,-2.21254095e-01,3.91151875e-01,2.78364927e-01,-1.49405599e-01,6.42492652e-01,-4.71754342e-01,-5.82797229e-01,5.89890685e-03,-8.44746530e-01,-2.33450338e-01,2.36510456e-01,8.23116452e-02,-6.68331206e-01,1.07987359e-01,-5.02974510e-01,-4.25702214e-01,-3.26761067e-01,-9.53198969e-01,1.68990746e-01,-1.25829160e-01,-1.78229645e-01,3.62343937e-01,-5.83929598e-01,-5.75413331e-02,5.31904757e-01,-2.49517396e-01,-5.51260933e-02,-8.33162010e-01,2.07786500e-01,-6.55504107e-01,-6.61204696e-01,3.52375895e-01,3.40332597e-01,-5.13430655e-01,1.10931301e+00,-1.76887542e-01,-4.50984091e-01,6.53890193e-01,-2.72161126e-01,-3.91595930e-01,-2.10009545e-01,7.62875751e-02,4.58191991e-01,3.31936777e-01,-1.10108519e+00,-4.11449634e-02,-8.94705415e-01,-3.28286618e-01,1.09662250e-01,-7.03248680e-02,1.35899812e-01,3.41456831e-01,5.73646784e-01,2.22085342e-01,-8.19487870e-01,9.98968557e-02,-7.22771883e-02,-8.87772068e-02,-5.91414869e-01,1.83583021e-01,4.72672969e-01,1.21352039e-01,-9.91822556e-02,-4.92472082e-01,-1.34347290e-01,-2.81579822e-01,4.08404559e-01,-4.39851969e-01,3.51319760e-01,2.07709819e-01,-8.16363752e-01,-7.99044192e-01,1.87352091e-01,-2.58156329e-01,4.40464467e-02,-1.92808419e-01,-4.79401499e-01,-1.17271528e-01,-7.41861045e-01,5.39903082e-02,-1.06682397e-01,6.54683113e-01,3.92834187e-01,1.67446598e-01,7.04000145e-02,2.53030747e-01,-7.19668150e-01,7.66898021e-02,-4.77423072e-02,-2.27601096e-01,-2.02404484e-01,4.77983132e-02,-5.51445410e-02,-2.52288043e-01,1.09955579e-01,-1.88720390e-01,-6.48304164e-01,3.85027915e-01,5.62586308e-01,-2.31441379e-01,-3.12054396e-01,1.28319427e-01,-8.46267343e-01,3.52430403e-01,-3.69988233e-02],[4.85917628e-01,-3.10527593e-01,5.72130919e-01,-7.24562168e-01,5.28699040e-01,-8.25683698e-02,-3.61857206e-01,-7.05154419e-01,1.39358744e-01,2.16667190e-01,1.79626957e-01,3.43648106e-01,3.90989035e-01,2.89416581e-01,-4.89554226e-01,-5.83800137e-01,-6.55240476e-01,-5.57561934e-01,-5.54121077e-01,7.11566567e-01,6.36228919e-01,-4.12617773e-01,-9.92789149e-01,-1.18771523e-01,8.00166652e-03,-2.50437617e-01,-3.46835017e-01,-3.45832765e-01,-4.99188632e-01,-4.02550623e-02,2.66084939e-01,3.34567636e-01,-3.13978314e-01,1.63653150e-01,-1.02224439e-01,7.52358615e-01,1.28710282e+00,-4.05252039e-01,2.16630533e-01,-3.62084031e-01,-1.24509084e+00,4.04730260e-01,-6.67072415e-01,-6.58896744e-01,4.02502358e-01,5.67833960e-01,-7.83283189e-02,1.84836611e-01,-2.71500528e-01,3.82259667e-01,-4.71359044e-01,8.34339708e-02,6.77084178e-02,-3.71103287e-02,6.19135499e-01,-3.57189983e-01,3.07732254e-01,-5.40038943e-01,3.35257620e-01,-1.61315531e-01,-2.42639348e-01,-1.74293265e-01,-1.04826674e-01,-8.88266787e-02,3.34105827e-02,6.02561355e-01,-1.11142635e+00,1.27011731e-01,-3.42531472e-01,-1.71907797e-01,-8.02048266e-01,-1.74155999e-02,1.34595439e-01,-2.94636656e-02,3.53935897e-01,8.23849082e-01,-2.80757308e-01,-2.15404898e-01,-3.18657570e-02,-2.33445484e-02,5.61853707e-01,9.21488088e-03,-5.38053811e-02,5.05724490e-01,1.19825609e-01,3.66466254e-01,-2.87404265e-02,-2.39249006e-01,6.27361655e-01,3.25026542e-01,5.60234129e-01,-1.36349827e-01,5.34506105e-02,4.32689115e-02,2.72663832e-01,-4.23182957e-02,1.64304361e-01,-3.68742526e-01,-6.04316235e-01,-2.18605325e-01,-2.21382990e-01,1.09753020e-01,-1.37130380e-01,2.80698121e-01,4.37070042e-01,6.77340850e-02,-3.13743204e-02,2.94523209e-01,-5.73175311e-01,-3.76969576e-01,-6.67795539e-02,-1.22502238e-01,9.20741618e-01,-1.13333143e-01,1.94649458e-01,4.00053114e-02,3.41349095e-01,1.59977321e-02,5.05898520e-02,5.29303253e-01,1.71851501e-01,2.33584762e-01,-8.58595669e-02,4.63386565e-01,-7.70822346e-01,-5.78113258e-01,1.43357694e-01,2.13781431e-01,-3.84586275e-01,4.17187989e-01,6.04568064e-01,3.67813200e-01,-6.95049942e-01,3.00720483e-01,1.58091277e-01,-2.14671224e-01,-1.12824053e-01,2.26560399e-01,2.77527571e-01,-4.67797548e-01,9.27956700e-02,-2.36508444e-01,3.84643584e-01,4.15732503e-01,1.38602749e-01,3.60507220e-02,-3.57621700e-01,-1.38451338e-01,5.31915009e-01,1.08734309e-03,9.40196856e-04,-1.45646306e-02,6.47104740e-01,5.14186919e-03,-4.70064655e-02,-5.34457387e-03,1.78208739e-01,-1.20283984e-01,5.19198000e-01,-2.94329613e-01,3.09710085e-01,-2.26999134e-01,-2.36089677e-01,-3.62612605e-01,-3.13620687e-01,-1.14727065e-01,-9.44567844e-02,2.70765156e-01,-4.41406220e-01,2.21331015e-01,-1.97764814e-01,4.23045665e-01,-7.16162860e-01,4.56138223e-01,-3.50203067e-01,-9.62859243e-02,-3.58813256e-01,4.42564338e-01,-9.05191377e-02,1.47771716e-01,-1.99956656e-01,3.54663014e-01,-1.77132860e-02,-6.28794849e-01,-4.27251756e-02,3.69871169e-01,6.23047829e-01,-1.13622800e-01,3.27594168e-02,6.74452707e-02,-2.27870978e-02,-8.85176100e-03,-5.87005913e-01,1.06843404e-01,6.21321082e-01,3.54200214e-01,-1.42915070e-01,3.19497794e-01,-4.76306351e-03,1.97872862e-01,-3.58840644e-01,1.47091106e-01,4.09615003e-02,-3.14329147e-01,-1.02317107e+00,3.33396822e-01,3.50429356e-01,-4.88476664e-01,-3.51985127e-01,-2.31567964e-01,2.98800617e-01,-5.32656789e-01,-9.38147828e-02,4.70618665e-01,-4.45585847e-01,-3.59035492e-01,3.35063376e-02,-7.65365422e-01,-1.25796005e-01,-3.35626870e-01,-3.90974969e-01,2.86550164e-01,-2.63416111e-01,2.48098210e-01,-4.71275449e-01,2.59093314e-01,-4.52550530e-01,2.17153475e-01,2.90493816e-02,1.14097875e-02,1.23472981e-01,-6.96013421e-02,9.60585177e-02,-3.46764177e-03,-6.70307815e-01,-5.98583043e-01,8.95402014e-01,-2.03701705e-02,-1.11157167e+00,6.21921062e-01],[-1.57929048e-01,4.20726508e-01,-5.04837394e-01,1.03623480e-01,1.76271781e-01,8.58937725e-02,2.78688949e-02,-5.39226234e-01,3.60998333e-01,-6.10430956e-01,3.14584017e-01,2.00101048e-01,-1.67971835e-01,2.09271058e-01,-2.70248145e-01,-4.04223025e-01,-5.11584841e-02,2.18280200e-02,4.27635983e-02,2.37220988e-01,6.14820480e-01,-4.75095153e-01,-1.09787464e-01,3.48092407e-01,3.02510500e-01,3.44231158e-01,-1.86865509e-01,-2.32626908e-02,9.61867943e-02,-1.80700123e-01,-5.21282375e-01,-1.00396417e-01,3.75910968e-01,-3.42364579e-01,7.52452552e-01,-7.93756843e-01,-7.47954607e-01,-1.16843253e-01,-4.48269062e-02,3.21148634e-01,5.27304411e-01,4.88439202e-03,-4.82865870e-01,3.73602629e-01,-9.29105561e-03,4.69082564e-01,-1.91614419e-01,-3.40470403e-01,-5.36506891e-01,-4.73687798e-01,1.86769649e-01,-2.31270820e-01,-5.29306471e-01,-1.01180606e-01,8.44801128e-01,-1.38909459e-01,-6.98722661e-01,2.37848714e-01,2.47993499e-01,9.17398557e-02,3.56783807e-01,-3.50093633e-01,-7.95977041e-02,-2.07902715e-02,-1.55121982e-01,2.61008590e-02,-2.07927212e-01,5.83901703e-01,3.84019732e-01,1.54465511e-01,-8.45103145e-01,3.39350943e-03,1.68182805e-01,-3.33490163e-01,2.18298495e-01,7.82208800e-01,3.65779102e-01,4.88257349e-01,-1.98052153e-01,5.03877103e-01,-2.27897614e-01,-2.87952662e-01,-1.12337470e-01,-5.94108403e-02,4.45953280e-01,1.07421264e-01,2.81818174e-02,-4.98034865e-01,-2.93109179e-01,3.68401915e-01,-1.17800176e+00,-3.66588265e-01,-7.93852732e-02,5.41786589e-02,-8.25296998e-01,4.52968538e-01,6.38150334e-01,-9.38236952e-01,9.62151662e-02,-1.28456801e-01,5.36277950e-01,-2.49953181e-01,-8.87029946e-01,-3.72832656e-01,-5.60018003e-01,-3.64192069e-01,8.87640864e-02,-4.02478009e-01,3.21908116e-01,2.55416304e-01,9.37918797e-02,-1.12452054e+00,1.32088029e+00,-3.41972500e-01,4.79542404e-01,-7.70757556e-01,-3.73356372e-01,2.54199028e-01,-2.74062842e-01,5.77171780e-02,-8.77564371e-01,1.25793052e+00,-8.37122798e-01,-1.73408985e-01,-2.95354694e-01,-1.37929344e+00,2.59966012e-02,-7.97773063e-01,-3.95485997e-01,9.57230106e-02,4.30977374e-01,-3.69863689e-01,3.24547976e-01,-2.82702506e-01,2.84338295e-01,4.98190612e-01,-1.93684638e-01,5.71146433e-04,-1.71684787e-01,-1.18367516e-01,2.02099338e-01,3.84039640e-01,5.54611444e-01,-6.46985078e-04,9.11293253e-02,1.11670569e-01,-1.55828476e-01,-1.29548032e-02,4.43286225e-02,-9.10450995e-01,3.16499710e-01,7.50930667e-01,2.44710863e-01,-1.91104114e-01,3.94430846e-01,2.69710839e-01,-1.76700577e-01,-5.82183838e-01,-1.37110725e-01,1.97012171e-01,-6.15747750e-01,1.37598887e-02,-6.37327850e-01,-4.65462029e-01,4.14600559e-02,-1.28285468e-01,7.39481524e-02,2.48426646e-01,-5.54059386e-01,2.66012818e-01,-9.64112207e-03,8.40024412e-01,-2.18563333e-01,3.01071942e-01,6.30067825e-01,-4.36916351e-01,4.25896704e-01,2.65849754e-02,1.78151220e-01,-5.59916615e-01,-1.44502506e-01,4.61834013e-01,-5.63707411e-01,-4.68102485e-01,-1.01957954e-01,-2.57409327e-02,-7.80679882e-02,1.65589735e-01,1.16578840e-01,3.23268622e-01,-1.04325019e-01,2.38224521e-01,2.31403280e-02,-9.54015136e-01,-5.72985888e-01,-2.12450117e-01,1.26183689e-01,-5.23340106e-02,2.36001667e-02,8.23021084e-02,4.43592757e-01,5.02732396e-01,3.37264597e-01,2.55522072e-01,3.32207114e-01,-9.05828271e-03,-4.61774439e-01,-3.41845840e-01,-1.69195145e-01,-4.40725207e-01,1.23962186e-01,3.69762331e-01,4.49132144e-01,-9.38240886e-01,2.79816121e-01,2.34703377e-01,3.66226077e-01,4.64950055e-02,-9.86685529e-02,-1.47151917e-01,2.97340184e-01,-5.21880209e-01,-2.25123111e-02,2.95932442e-01,-4.24075246e-01,2.32024089e-01,-1.02165975e-01,-1.18036037e-02,-1.26459710e-02,-7.30121061e-02,-4.51578170e-01,2.82728583e-01,4.88558374e-02,1.06734134e-01,-7.78361335e-02,-2.10507780e-01,2.63284713e-01,-2.67575562e-01,3.38417828e-01,-5.80875397e-01],[3.04110080e-01,3.17967921e-01,-2.15992153e-01,5.00877127e-02,-1.79261230e-02,-6.95537269e-01,1.98830411e-01,6.56745508e-02,-2.32989937e-01,6.74675524e-01,9.92889285e-01,-5.86357594e-01,1.08605377e-01,-5.89198887e-01,-1.03854924e-01,-1.04457155e-01,3.35095674e-01,-3.56653005e-01,-1.08211219e-01,-5.75950027e-01,-3.18571657e-01,-7.97142208e-01,-3.98230344e-01,-6.47331655e-01,-7.57970810e-01,7.43960142e-01,-3.43722939e-01,8.88615310e-01,3.00193783e-02,7.49161243e-02,2.32169136e-01,3.89149547e-01,-2.59226441e-01,1.11111784e+00,-8.17948401e-01,-2.48717025e-01,4.29779366e-02,-1.69116318e-01,-3.66195738e-01,-1.23761013e-01,6.13754570e-01,-7.30283499e-01,3.91457051e-01,-6.05708122e-01,-1.93385839e-01,-1.97263807e-01,2.63383895e-01,-2.34504297e-01,9.72834527e-02,4.89792049e-01,-2.87992984e-01,-1.04614161e-01,1.98830202e-01,-1.01338744e+00,3.35389227e-01,-9.54056308e-02,-7.19443202e-01,-7.14180097e-02,2.82981753e-01,-1.88855186e-01,2.14109361e-01,-4.59121317e-01,3.81764889e-01,4.99219038e-02,2.82528311e-01,1.25350699e-01,8.46493423e-01,-4.45926450e-02,-9.81568396e-02,-2.81037003e-01,-5.25663495e-01,-2.57101506e-01,-4.16143894e-01,-6.06048144e-02,-6.21869028e-01,-5.31843364e-01,3.78721774e-01,-5.21876402e-02,1.30725339e-01,9.28101614e-02,2.10220933e-01,-1.78921789e-01,-1.21979252e-01,1.25363961e-01,-4.39830244e-01,4.64227140e-01,-2.85630047e-01,4.10277486e-01,-8.32501590e-01,-1.55779749e-01,1.06466830e-01,4.73970920e-01,1.44476844e-02,5.84430337e-01,7.39385664e-01,3.47607791e-01,-3.18241090e-01,5.73808253e-01,-3.45231563e-01,9.33203474e-02,4.77405965e-01,3.70710284e-01,4.46023107e-01,4.30558622e-01,-4.34345484e-01,-8.74265730e-02,-3.16962361e-01,-8.32236037e-02,4.95703459e-01,-3.03398639e-01,-1.80453762e-01,-1.25004604e-01,4.64549452e-01,-2.09777504e-01,3.21429670e-01,-7.89466918e-01,6.89841211e-02,-5.65699767e-03,6.46371722e-01,4.99866670e-03,1.30829409e-01,6.38553500e-01,2.59174824e-01,-6.76546276e-01,-2.92118549e-01,-4.53176821e-04,5.47442555e-01,-3.15408617e-01,-1.77379325e-01,1.28307939e-01,-5.59795141e-01,1.49137080e-01,-6.24820948e-01,1.19361848e-01,2.91360449e-02,-8.50036561e-01,-4.44502026e-01,1.12764135e-01,-2.07876116e-01,-3.13544452e-01,1.22067004e-01,-6.86083972e-01,-6.81312978e-02,4.87468511e-01,-5.56188047e-01,-6.67133033e-01,5.82400441e-01,4.17543799e-01,1.88108176e-01,1.10386088e-01,-1.78873062e-01,5.64087510e-01,1.90645933e-01,-3.33349228e-01,-6.33017361e-01,9.56998542e-02,3.25685561e-01,-5.19320741e-02,5.12862504e-01,-5.66048384e-01,-5.34074008e-01,9.04689372e-01,1.59943596e-01,4.49164420e-01,5.37653029e-01,2.25840122e-01,-5.03178060e-01,3.52548897e-01,-7.02490509e-02,-3.95101786e-01,2.13018153e-02,1.14738971e-01,-1.28153220e-01,-3.59206609e-02,3.32776904e-01,-1.80593655e-01,7.67218471e-02,-2.16769889e-01,-9.63534117e-02,-3.09766885e-02,-1.49672776e-01,-1.03883430e-01,2.58887231e-01,3.00271183e-01,-3.76508266e-01,-1.15215190e-01,-3.08263928e-01,7.73578584e-01,4.97242540e-01,-1.24143269e-02,-7.94462204e-01,-4.52574909e-01,-9.89269435e-01,6.18781708e-02,-8.36781110e-04,5.81430316e-01,-1.08511376e+00,5.29471897e-02,-1.37560889e-01,2.53856361e-01,-3.83496881e-01,-9.00020897e-01,2.52985954e-01,-8.36174116e-02,-3.08680743e-01,1.97015136e-01,-8.09484497e-02,-5.45282900e-01,-6.99564874e-01,4.62326169e-01,1.51829585e-03,-1.77708820e-01,-7.59468526e-02,4.58574921e-01,-9.76141840e-02,2.47689560e-01,-6.74149573e-01,-2.22266018e-01,-3.58538449e-01,-1.33942440e-01,-4.32167649e-01,4.79509681e-02,6.97500110e-01,-6.61141574e-02,-4.59822148e-01,-6.07101679e-01,4.14665848e-01,-9.27488208e-02,6.76603019e-02,-2.09707543e-01,-4.20747519e-01,-4.28677708e-01,-7.66212270e-02,3.96407038e-01,-1.49504915e-01,-7.17124715e-02,1.11535299e+00,3.46418500e-01,4.09761608e-01,-4.27017778e-01],[1.74927711e-01,-1.31407142e-01,6.93129078e-02,-2.01258019e-01,-4.67864543e-01,-8.08463320e-02,-3.49966466e-01,-5.65224171e-01,-3.88082445e-01,-3.31590682e-01,3.94977063e-01,6.31953357e-04,8.76265615e-02,-2.59914905e-01,6.58765137e-01,3.40656608e-01,-5.93268573e-01,-5.92842847e-02,5.49391270e-01,2.59699970e-01,3.20529103e-01,2.05455944e-01,-1.23646155e-01,7.40616545e-02,-3.40719223e-01,-3.29919904e-01,-6.14905715e-01,-5.18426076e-02,1.35836601e-01,3.92238349e-02,2.46698670e-02,2.89979726e-01,-3.69874984e-01,2.76318014e-01,-7.94050023e-02,2.74098486e-01,3.23333144e-01,-1.66352950e-02,-9.88344625e-02,-2.30091363e-01,-6.56963885e-01,-2.74598271e-01,1.19498663e-01,6.92826211e-02,-6.69388711e-01,-1.86341032e-01,7.96328247e-01,6.85753226e-02,-3.32955688e-01,8.25014152e-03,2.74538338e-01,-1.21187434e-01,-6.23049676e-01,-1.01660594e-01,4.78478223e-02,-1.06615715e-01,-3.60450655e-01,8.98190141e-02,9.56407115e-02,-5.35575986e-01,-2.92688888e-02,1.77631706e-01,2.69542396e-01,-5.76567113e-01,-5.65063834e-01,-1.65831804e-01,3.49221438e-01,7.24476218e-01,2.26767242e-01,-2.81210124e-01,7.45478451e-01,2.81076610e-01,-6.91573858e-01,9.14146528e-02,2.65236437e-01,8.78713965e-01,1.54753760e-01,1.19285733e-01,-4.78717208e-01,8.43871832e-02,8.44016075e-02,-1.10043362e-02,2.68338054e-01,4.04889584e-02,4.31474626e-01,5.85592934e-05,-7.82703161e-02,8.80363956e-02,-2.62892008e-01,4.80364710e-01,1.11364722e-01,1.68890044e-01,-2.55467921e-01,-6.61760867e-01,4.14140016e-01,-3.53084236e-01,2.47289464e-01,3.51488054e-01,-1.02927476e-01,1.27700537e-01,-2.42720515e-01,3.00606251e-01,-7.29971975e-02,3.33692819e-01,1.08902681e+00,3.12567949e-01,1.53559580e-01,-1.52279243e-01,9.13340032e-01,4.11770344e-01,5.99864304e-01,4.39727634e-01,-4.72133040e-01,-5.59408903e-01,-7.84822345e-01,-3.36280018e-01,4.13886875e-01,3.67431715e-02,2.66721398e-01,9.53022540e-02,-3.02200526e-01,2.82378048e-01,-4.10768569e-01,-4.21884358e-02,3.68275821e-01,2.04740405e-01,-2.11642891e-01,1.93026990e-01,2.28410587e-01,-1.29550070e-01,8.39927420e-02,-1.07128561e-01,-6.81808114e-01,1.04157284e-01,2.95398265e-01,-5.49001813e-01,3.77857238e-01,1.81271210e-01,9.22762871e-01,-4.51415539e-01,-5.68373241e-02,-1.84826165e-01,2.12260082e-01,-1.80855691e-01,-9.93687451e-01,1.50300026e-01,3.85966867e-01,1.95199043e-01,1.02462888e-01,5.47360599e-01,5.85727930e-01,1.75630286e-01,6.38277888e-01,6.88853636e-02,2.83337653e-01,5.06622996e-03,4.45801437e-01,-6.36372626e-01,1.53544962e-01,-2.82702222e-03,6.71425700e-01,-8.06959867e-02,8.04900169e-01,-1.73644841e-01,-1.72155172e-01,8.07065547e-01,3.86408359e-01,-7.43848532e-02,-6.25720441e-01,-3.00302625e-01,-3.79671752e-01,-5.01248479e-01,-1.33383587e-01,-6.53983802e-02,-3.94566506e-02,1.25963584e-01,-4.05836105e-02,1.33014008e-01,-9.15754139e-01,-2.87764426e-02,-1.01954713e-01,-1.60763562e-01,1.29081875e-01,-2.53278494e-01,-2.47319788e-01,4.12471354e-01,4.57088500e-01,4.46642302e-02,3.67336534e-02,-1.20662153e-02,4.11208212e-01,2.09433600e-01,1.00209869e-01,-1.02401756e-01,-5.64129114e-01,-9.64109078e-02,5.97469985e-01,1.01105519e-01,-6.23456538e-01,-1.31257456e-02,3.81279260e-01,8.10616314e-02,-3.04239020e-02,-1.59015283e-01,-4.69040513e-01,4.46998656e-01,8.02964047e-02,-2.90707201e-01,-6.90380409e-02,-7.70524740e-02,-7.81208515e-01,5.02905428e-01,6.97501838e-01,-1.54206917e-01,1.97385311e-01,2.84476906e-01,1.06300339e-01,4.66168940e-01,1.73829466e-01,-1.95455030e-02,-9.74660158e-01,-1.43165849e-02,5.48358150e-02,-5.29171936e-02,1.50667250e-01,-2.13833153e-01,1.53032839e-01,5.84739804e-01,5.87385178e-01,2.27850918e-02,3.05356890e-01,5.28960586e-01,-6.22982830e-02,2.70574018e-02,4.37783927e-01,2.92252958e-01,-8.20610076e-02,-6.40430808e-01,3.92546058e-01,5.05133718e-02],[0.09660903,-0.14097084,0.44302404,-0.10182431,-0.63195986,0.34755594,-1.1458125,0.22660874,0.14271396,-0.25183743,-0.43093547,0.0684538,0.05280638,0.209113,0.47676903,0.31210864,0.06823789,-0.5274913,-1.0045521,-0.08116665,-0.05433515,-0.388747,-0.48669758,-0.20748563,-0.35271826,-0.09868683,-0.8504471,-0.87843096,-0.44126928,0.46569046,-0.00811139,0.5550558,0.17334005,-0.12855755,0.0906034,0.30875623,0.05690601,-0.3601557,0.18049285,-0.19513632,0.05262882,-0.33346814,-0.5255489,-0.27195913,0.5572,0.48106837,-0.27341035,0.06747697,-0.30442527,0.07786474,-0.07572429,0.35080034,-0.01063356,-0.26124892,0.60105884,-0.52166337,-0.06351519,0.24211481,0.77676535,0.721564,-0.13990118,0.28892177,-0.13900836,-0.01534081,-0.24509491,-0.10272104,-0.5060935,-0.30851382,-0.07988245,0.05826112,0.19820206,-0.06487171,-0.21101096,-0.390521,-0.00145492,1.1571008,0.3926251,-0.06724369,-0.50593835,0.31636244,-0.00291215,0.42736667,-0.07404301,0.22570924,-0.02104071,-0.06849473,0.30050305,-1.3070685,0.1055515,-0.04161938,-0.03440516,0.03899958,-0.21856071,-0.03949792,-1.0303679,0.05434857,0.43136203,0.03597378,0.25991476,-0.1984675,0.44252065,-0.5618979,0.39294392,-0.11538084,-0.11876013,0.2036728,-0.46548623,0.62179494,0.38292828,0.09340804,-0.31401038,-0.27840936,-0.28120464,0.37129894,-0.24710011,0.09626059,0.17777807,-0.17683071,0.1931054,-0.65047365,-0.29124966,0.33938923,0.06165762,-0.6710782,-0.92800057,0.10657341,-0.20108627,-0.44086182,-1.1645416,0.12347036,-0.7817497,0.55029815,-0.07101726,0.64992946,-0.15075423,0.28539127,0.6482898,-0.19498223,-0.5212252,0.40521076,-0.24014573,0.0296609,0.19051255,-0.14000173,0.34409308,-0.3792591,0.22479825,-0.43655738,0.13485955,-0.11719242,-0.01957123,0.56997484,-1.0338389,-0.02034051,0.18098962,-0.5470483,-0.7372697,-0.01358441,0.20927636,-0.38616934,0.12672915,0.30563244,0.13856755,-0.60581005,0.5478571,-0.02447719,0.1127632,0.2012633,0.46569467,-0.1249804,0.31242022,0.08285431,0.20402846,-0.09392022,-0.6789574,-0.16814455,0.9309585,-0.31073418,0.18755993,0.08057709,-0.10849658,0.11352798,0.03931277,0.04946803,0.27196848,0.14955425,-0.28357628,0.25867507,0.7392838,-0.7550423,-0.5181515,0.49199477,-0.7301791,0.27561638,-0.47075924,-0.304281,0.49548692,-0.04938663,-1.1497666,-0.38349447,-0.18967004,-0.61084104,0.36146381,-0.44749638,0.20568925,0.06326678,1.1141049,-0.34852442,0.8114919,0.2518631,0.2523385,-0.61542344,0.65477335,-0.01803346,-0.6579563,0.14707507,0.49242988,-0.770974,-0.01329332,-0.9881478,0.5011979,0.66069764,-0.4952383,0.36281964,-0.13164797,-0.15476184,0.22319368,0.2879755,1.186486,-0.17061023,-0.43258992,-0.36411068,-0.22027105,0.35070327,-0.20372307,0.33066738,-0.44538316,-0.07347377,-0.15047082,-0.16949758],[5.77125400e-02,4.50045355e-02,-1.59939565e-02,6.01828247e-02,-3.86015683e-01,7.23787174e-02,-5.21654189e-02,1.42463101e-02,4.47497107e-02,4.61444646e-01,2.33147308e-01,-1.96828485e-01,4.07797426e-01,-9.93533254e-01,-7.95638934e-02,1.86923593e-01,1.36571407e-01,3.82934809e-02,-1.40674338e-01,-4.80964243e-01,-4.11984861e-01,-3.22060257e-01,-6.17548048e-01,2.09582195e-01,-4.02325422e-01,9.31149274e-02,1.12965949e-01,1.20732702e-01,-6.88719213e-01,-1.50827646e-01,-8.43087196e-01,3.60374928e-01,3.96558970e-01,6.64804652e-02,1.03208959e+00,4.95956391e-02,5.35009146e-01,-7.44308755e-02,-3.80404234e-01,-3.79558384e-01,-1.21173310e+00,5.42944312e-01,5.91201425e-01,8.08214366e-01,-3.49859148e-01,2.78646171e-01,-4.04266328e-01,2.39059925e-01,-1.64613500e-01,-7.27536440e-01,6.26937971e-02,1.38197362e-01,-3.00594326e-02,-1.15501836e-01,7.36622810e-01,-5.76792955e-01,-1.67022515e-02,1.01646475e-01,3.06680620e-01,2.63229281e-01,-3.78585637e-01,4.85904694e-01,-1.07194863e-01,3.12985592e-02,-5.76862991e-01,1.22273996e-01,3.87029946e-02,-9.51809660e-02,-1.09113559e-01,-1.23093992e-01,-4.27770726e-02,3.83832008e-02,-2.07457379e-01,5.06462514e-01,8.39573145e-02,-3.16906631e-01,2.82374769e-01,-3.28169107e-01,3.14158469e-01,5.39151467e-02,2.58852750e-01,-6.27139628e-01,4.63615716e-01,-3.73897672e-01,-2.29032617e-02,8.88736174e-02,5.87000728e-01,8.47432613e-02,-2.83030063e-01,3.71760547e-01,-2.60646760e-01,-4.19180006e-01,5.46988845e-02,5.00499606e-01,4.05883253e-01,-3.35785568e-01,-8.23884830e-02,-8.62662911e-01,2.90293425e-01,-2.36299127e-01,-3.91835682e-02,6.38149381e-01,-7.03843296e-01,1.69784985e-02,3.67134124e-01,2.64264166e-01,4.78170067e-01,-6.78428486e-02,6.52775347e-01,-6.93707913e-02,-1.26998231e-01,2.27987304e-01,-4.61713821e-01,4.56614494e-01,9.17571336e-02,2.53411140e-02,-1.81219622e-01,4.41614985e-01,1.09074645e-01,3.58388752e-01,2.87100881e-01,7.80079961e-02,8.35945249e-01,-4.24226433e-01,-2.77866542e-01,6.95182502e-01,-3.34190935e-01,2.64136195e-01,7.16401935e-01,-3.14879954e-01,2.82553226e-01,3.64171267e-01,-4.56779301e-01,1.40433460e-01,3.29805791e-01,-1.58790301e-03,4.32983220e-01,1.71737894e-01,2.23901033e-01,7.61124045e-02,-9.31116045e-02,-4.91001576e-01,-5.86334229e-01,4.36743617e-01,7.41983950e-02,1.29945040e-01,4.95313942e-01,-3.82917106e-01,2.71531075e-01,-6.74505353e-01,-2.91740447e-01,1.06611812e+00,-4.65929061e-01,-8.95583034e-01,-1.56241000e-01,5.30800939e-01,9.95782763e-02,4.62840915e-01,-4.81071770e-01,-3.88383418e-01,7.26543665e-01,-1.27545342e-01,-3.05951647e-02,3.18825766e-02,1.19442262e-01,1.82566211e-01,8.97860005e-02,4.38268512e-01,-7.77683020e-01,-3.71803403e-01,-9.43502784e-01,1.37432385e-02,-2.21131250e-01,5.59867583e-02,-3.17721933e-01,-3.89316589e-01,5.92905395e-02,-9.66393426e-02,-1.71753392e-01,-3.02654505e-01,1.83903113e-01,4.13530558e-01,2.26449162e-01,-1.23656839e-01,2.59411484e-01,-6.44999206e-01,-7.71762967e-01,3.22619528e-02,5.82827032e-01,2.65542477e-01,-6.69968963e-01,-5.25600195e-01,6.78352892e-01,-4.77587223e-01,2.77746320e-01,-6.46147549e-01,-5.35867512e-01,-1.55069783e-01,-1.61605462e-01,-3.54052931e-01,-1.41006976e-01,-2.57058352e-01,-2.28872553e-01,-3.41966808e-01,3.65671724e-01,-5.82067907e-01,4.74153370e-01,9.72803682e-02,4.20628153e-02,2.20229760e-01,5.18674217e-02,-5.01056254e-01,-8.13240826e-01,-4.19376552e-01,2.47854561e-01,-3.00971299e-01,-6.45779371e-02,5.27145788e-02,-4.89072829e-01,5.59084654e-01,-7.05075383e-01,-3.47487003e-01,-4.12143290e-01,5.87618470e-01,-1.12429723e-01,7.76786881e-04,-2.65302449e-01,-2.18955465e-02,-5.54670990e-01,-3.73959273e-01,9.06720698e-01,-7.72263184e-02,-3.79994184e-01,-8.70235413e-02,1.22521050e-01,3.51636931e-02,-2.92312473e-01,-9.94377613e-01,-3.88413593e-02,-3.60533223e-02],[-0.22964178,-0.63659805,0.5197315,-0.38292825,-0.61013526,0.00253217,0.35794368,0.05244014,-1.0007176,-0.54118013,-0.25247592,0.16911054,-0.20551315,0.44365516,0.34730574,-0.12585291,0.17844495,-0.4602143,-0.48757935,0.09404508,-0.18998697,0.2872359,-0.07395027,0.33418053,0.635471,0.29648107,-0.32530847,0.17843851,0.45322552,-0.12141985,-0.44068313,-0.25446615,-0.24275658,-0.03550637,0.22694488,-0.3063562,-0.0012119,-0.69812435,0.06015056,-0.34768704,-0.9889686,-0.61877096,-0.60813713,-0.3636982,-1.0252806,-0.48212948,0.40822938,-0.28139687,-0.04162183,-0.23731488,0.72106636,0.18118094,-0.6602584,0.556636,0.26558316,-0.35339698,0.02811807,-0.1114124,0.5870001,0.73276484,-0.4047943,-0.16433243,-0.06339683,-0.2260811,-0.6337577,0.2821614,0.49156806,0.28488094,0.5737138,0.5121195,0.78000313,-0.00296822,-0.48000246,-0.07603952,0.09259481,0.2944221,0.2851784,-0.24254975,0.23141828,-0.3152789,0.23534448,-0.41658667,0.32400626,0.07090753,-0.10648271,-0.44513208,0.14212333,0.5100422,-0.25356168,-0.08672865,0.3393218,-0.18485469,0.4253935,0.17520279,0.2959305,-0.07404008,0.11135942,-0.625841,0.04232192,-0.4510268,0.06831864,0.03290256,0.01089745,-0.08034416,0.42390177,0.44760558,0.08277618,0.12116227,0.6606905,0.7089051,0.48738983,-0.42076212,-0.03483607,-0.5118954,-0.15464838,-0.76784027,0.60615134,-0.7270611,-0.03478584,0.50849056,0.366633,-0.01322947,-0.02795219,-0.31444103,-0.29105842,0.24687202,-0.437548,-0.32957146,-0.37340513,-0.5646628,0.43037933,1.0750226,0.68121856,-0.47990695,0.9319049,0.11342836,0.34120268,0.09075198,0.31042528,-0.38312867,0.14117959,-0.48695472,0.65862644,-0.10591176,0.07376359,0.42432255,-0.04349135,-0.02934068,-0.25417188,-0.24923651,-0.2171118,0.04034252,0.07742281,-0.4464147,-0.06344852,-0.19221959,0.01486791,-0.09213144,0.16040504,0.58771855,0.5211522,0.3435783,0.23082213,0.28164843,0.36365172,-0.07142244,-0.6997251,0.31961435,-0.6665762,0.5972426,-0.5582728,0.31605732,-0.5234577,0.17340101,0.36535132,-0.16556777,-0.30805835,0.05197128,-0.08619396,0.2943605,0.12094806,0.48113275,0.24982128,-0.717202,-0.2032829,0.08483546,0.48348904,0.20706867,0.30305037,0.06149769,-0.05555348,0.06375224,0.36014622,-0.30053663,0.02316629,-0.8279703,-0.29642323,0.41697687,0.0900849,0.33109564,-0.09182773,-0.3436128,0.05832778,0.6304648,0.13079785,-0.06799034,-0.8030197,0.3134801,-0.4517054,0.7432307,0.6764063,-0.3069723,-0.21721038,-0.07327993,-0.17550512,-0.17583065,-0.6069816,-0.782476,0.5992969,-0.33580226,-0.11993767,0.70707566,0.17605373,-0.24219164,-0.48022383,0.1175553,-0.77996135,0.25131926,-0.21921967,0.30326846,0.39461204,-0.80732894,-0.48957843,-0.38245553,-0.06310317,-0.3033391,0.3637061,-0.6248717,0.09897976,-0.5433458],[-0.12596366,0.01603419,-0.44530207,-0.44650814,-0.33816478,0.02833612,0.05718391,0.13096969,0.7125346,-0.00311097,0.47023508,-1.0714073,0.45790792,0.20618282,-0.43816158,0.20002806,-0.34073192,0.05641445,0.48316032,0.5183945,0.7531353,0.50280744,-0.31181973,0.1863046,0.23778597,-0.11513255,0.14059113,-0.6670231,-0.4942498,0.62717104,-0.20137227,0.14623243,-0.67199636,0.12263101,0.82485825,-0.62338585,-0.14761926,-0.05241576,0.4577589,-0.52317697,-0.3048525,0.3435789,0.17376252,-0.25681287,-0.6172595,0.3231009,0.42812425,0.28273445,-0.65674764,-0.21542634,0.19343367,0.3499229,-0.09266443,-0.17204307,-0.33239016,0.19852023,0.04476501,0.63551843,-0.03249613,0.78584653,0.20070094,-0.10677271,-0.4464888,-0.33146265,-0.2786042,0.54481035,-0.3511772,0.30851257,-0.72272456,0.6151994,-0.42507327,0.5984037,-0.16943265,0.5343678,0.3578903,-0.04469173,-0.0223152,-0.17994349,-0.29511276,0.3135675,0.22214256,-0.17428604,-0.0566799,-0.3745857,0.24531609,-0.71959335,0.10200487,0.00988662,0.51799166,0.3387227,0.03825393,-0.65343374,0.41843754,-0.30575418,-0.16219665,0.07628342,-0.0113098,-1.0116018,-0.18051454,0.18923114,-0.22972342,0.89602506,0.8440115,-0.0739193,0.01168161,-0.0687291,0.15073842,0.1893567,-0.3693536,0.21730329,0.10389677,-0.44365397,-0.2579336,-0.03843755,-0.38299832,0.7904055,-0.30871966,-0.0214033,0.40931538,0.1467732,0.13876924,0.040125,-0.68720067,-0.33610687,0.29743668,-0.33122465,0.22727036,-0.02876256,0.47832024,0.01453905,-0.28695327,0.02650442,0.3413833,-0.42971325,-0.14757484,0.11505187,0.9692393,-0.80438215,-0.00340921,0.44040614,-0.8131102,0.53655297,-0.33135462,-0.46860453,-0.13121949,0.26767838,0.13381767,-0.17892358,0.03315891,-0.5318041,-0.22007197,0.21401986,-0.16113444,-0.02714172,0.27524197,0.01392619,-0.7937512,-0.6997619,-0.71376073,-0.12550402,-0.40285522,-0.62906367,0.15645055,0.03352796,-0.70106614,0.19609186,-0.10868044,0.18186961,0.55734134,-0.20750244,0.00350413,0.11251224,-0.14810139,0.331859,-0.6032825,0.19664693,-0.17862551,-0.21944569,0.04165313,-0.10697369,0.33180228,-0.27039894,-0.3192304,-0.67544377,-0.14510122,0.11031649,-0.3447589,-0.81008965,-0.33890614,-0.16089611,-0.1288512,0.08361567,-0.10230999,-0.4097593,0.35484135,-0.3379016,0.03019002,0.8298215,1.1485916,-1.0632573,0.3753764,0.31138527,-0.5258465,0.32174745,0.24135587,0.28211462,-1.097297,0.37754697,0.5515178,0.36271018,-0.74531984,-0.09565814,0.504968,0.30480444,0.7033227,-0.0373643,0.4391583,-0.30828708,-0.44722086,0.05254948,-0.6681359,0.3471515,-0.8882639,-0.09494169,-0.02901232,0.20181698,-0.12994994,-0.58134186,-0.29985368,-0.28917417,0.27193868,-0.8905566,-0.03298528,-0.20024604,0.59627545,-0.1664317,-0.19499408,-0.6130854,-0.21527046,0.24721348],[-2.35930622e-01,1.06657773e-01,1.78960800e-01,2.00901702e-01,4.21739608e-01,-6.28745675e-01,2.55244732e-01,4.77573760e-02,-1.67554736e-01,-6.65061027e-02,3.39290649e-02,2.32221097e-01,-3.13525200e-01,-3.37158412e-01,-7.13892207e-02,1.32394597e-01,-9.06135887e-02,-1.83812931e-01,-5.22953868e-01,-4.88533169e-01,2.69808650e-01,-1.26452708e+00,-3.95206772e-02,-2.95017809e-01,2.88565218e-01,3.02991450e-01,1.74938440e-01,1.02964008e+00,-3.48272771e-02,3.31547558e-01,3.95726055e-01,-6.75970614e-01,-6.34906664e-02,-4.22198683e-01,-3.71526659e-01,-4.04211789e-01,-1.59658387e-01,-1.29358426e-01,-8.75036195e-02,-8.52032974e-02,2.32512414e-01,1.95637092e-01,5.10814726e-01,5.27155936e-01,3.35442036e-01,4.80434716e-01,-3.39763641e-01,-2.73695409e-01,2.26489231e-01,-2.67013282e-01,-4.08971280e-01,4.13452163e-02,1.98280141e-02,-2.31550112e-01,-5.86383119e-02,6.34907722e-01,4.54494298e-01,7.60068297e-01,-2.14749604e-01,-6.16775453e-01,4.71162111e-01,1.21145062e-01,-4.65859137e-02,4.41999920e-02,3.63205880e-01,-4.54730093e-01,-2.42711440e-01,1.73375517e-01,-5.25767989e-02,-1.55536056e-01,2.15261489e-01,1.28672525e-01,-3.73062789e-01,-1.93201035e-01,1.46102175e-01,-3.00505430e-01,3.78112912e-01,9.57346112e-02,1.49802908e-01,4.08217579e-01,8.34615678e-02,4.79256570e-01,-5.79681695e-01,2.08229825e-01,2.64356434e-01,1.49626657e-01,-2.77635694e-01,-1.32478699e-01,-6.80915192e-02,1.17903231e-02,6.09672427e-01,5.63407958e-01,-3.42876464e-02,-4.52350974e-01,1.29069597e-01,2.63679832e-01,4.48910743e-02,1.82707515e-02,9.93099362e-02,5.62033415e-01,2.63496220e-01,2.97456402e-02,-2.49098063e-01,3.74930650e-02,2.35170461e-02,4.66858387e-01,-2.57857710e-01,3.55856955e-01,4.85711657e-02,-2.35316887e-01,-7.48142451e-02,3.12274005e-02,-1.02218129e-01,7.98740089e-02,5.87700084e-02,-2.91800220e-02,-4.44121689e-01,-4.34547663e-01,9.49140042e-02,7.99194098e-01,3.25113952e-01,2.56849766e+00,-2.29764998e-01,-1.74385726e-01,1.48422942e-01,-1.32701069e-01,-6.44984916e-02,6.67279363e-02,2.52717167e-01,-1.01821852e+00,-9.81171709e-03,1.57166924e-02,5.23158669e-01,5.09960413e-01,-4.24278200e-01,7.36662090e-01,-4.05374527e-01,1.76039234e-01,2.83645749e-01,-4.93175209e-01,1.98246196e-01,-3.19614075e-02,-1.96784198e-01,-1.43575937e-01,-1.21803246e-01,-4.03642118e-01,2.78307926e-02,-1.23960085e-01,5.75756907e-01,-1.11586720e-01,2.75173515e-01,4.33269560e-01,2.53790528e-01,-5.25341853e-02,-6.49031550e-02,1.23925216e-01,-9.92253423e-02,1.02485800e+00,4.90488969e-02,7.58454651e-02,-8.45364556e-02,-6.38910234e-02,1.85715392e-01,5.88164210e-01,-2.46632937e-03,4.25502807e-02,-4.13493253e-03,1.44272298e-01,1.37681335e-01,4.10228968e-01,-7.51897395e-02,-1.16059616e-01,1.11335358e-02,-2.38579214e-01,2.81230686e-03,-3.95474076e-01,2.89242655e-01,-7.32317865e-01,4.42936122e-01,-1.20916724e-01,-4.76563960e-01,-3.19965512e-01,5.36536649e-02,-3.07682127e-01,5.76949060e-01,3.18690389e-01,1.94690108e-01,3.20133299e-01,-4.42282967e-02,3.54117781e-01,1.67206913e-01,2.45497599e-01,1.49095044e-01,1.48184925e-01,2.51022071e-01,5.47157109e-01,-3.14181391e-03,2.98322648e-01,-3.30923617e-01,3.68534327e-02,3.36295888e-02,-4.08027351e-01,3.31907153e-01,1.58683479e-01,-1.28610373e-01,1.70733437e-01,3.07745457e-01,3.07645589e-01,3.60424817e-01,3.47089559e-01,-1.88696802e-01,2.83991452e-02,-2.43655190e-01,5.25266171e-01,-1.40282094e-01,-4.57257062e-01,3.17451358e-01,8.23689818e-01,1.51742706e-02,2.99971461e-01,-7.22349137e-02,-1.99569523e-01,1.18407138e-01,2.77932137e-02,4.36945528e-01,4.36545104e-01,-8.48036706e-02,2.50796109e-01,-4.45850909e-01,-5.94962120e-01,-3.37082386e-01,1.94060773e-01,-3.88015479e-01,-4.33434248e-01,2.26970404e-01,-2.59649217e-01,-7.80548453e-02,-9.30389702e-01,1.92211010e-02,-3.18007916e-01],[2.02288285e-01,4.17569518e-01,8.06912005e-01,1.57461569e-01,-6.29382670e-01,6.28255785e-01,2.58750558e-01,-1.49056055e-02,-5.86166620e-01,-3.09030712e-01,-8.68434072e-01,1.97935253e-01,2.85514593e-01,-1.05968164e-03,-8.31267536e-02,6.49896681e-01,-4.56735432e-01,-4.87488151e-01,-6.04482591e-01,-1.52013540e-01,1.53614193e-01,-5.70444465e-01,-1.22059412e-01,-6.44742399e-02,-1.00517452e+00,2.82734156e-01,7.96898231e-02,-6.57814085e-01,-7.71255434e-01,-7.65121728e-02,-7.41357744e-01,3.73422094e-02,1.99853197e-01,-4.06147987e-01,4.59158540e-01,3.60390693e-01,8.69818449e-01,4.72134918e-01,2.66510904e-01,4.65976834e-01,-8.59058082e-01,-1.60704195e-01,-1.24652408e-01,2.87431255e-02,1.38611256e-04,-2.78580248e-01,2.82678246e-01,-3.80730242e-01,-6.90123320e-01,-8.93615559e-02,1.42650887e-01,2.52377421e-01,-1.44507790e+00,-5.53630233e-01,7.06702843e-02,-9.73546132e-02,1.28261641e-01,-1.67250842e-01,-1.23450287e-01,6.79916143e-02,-1.26573786e-01,1.57274321e-01,-2.65097737e-01,1.58571124e-01,-1.67670563e-01,-8.57788995e-02,1.13944340e+00,2.57438660e-01,5.65435231e-01,7.97822103e-02,1.19722970e-01,5.30949607e-02,-9.92464066e-01,-3.74105096e-01,-9.21756804e-01,-2.44313985e-01,1.95868924e-01,2.44175240e-01,1.37828991e-01,-2.54736185e-01,2.30250448e-01,2.37770781e-01,6.28742576e-02,-1.42015979e-01,1.82106301e-01,-2.58573860e-01,-2.11620644e-01,3.42706949e-01,-5.74269518e-02,-1.94356009e-01,-2.10027546e-02,3.19769293e-01,-1.01248279e-01,-8.59809220e-02,-6.74057379e-02,3.37087780e-01,2.40601376e-01,-4.48140740e-01,4.13446724e-01,-4.41810966e-01,6.05341122e-02,1.08782560e-01,-7.17919469e-01,-2.38070384e-01,1.71735954e+00,2.82489151e-01,-3.42455268e-01,4.69694585e-01,1.90384567e-01,-3.79081853e-02,8.55899692e-01,4.65240717e-01,5.23565561e-02,3.34132493e-01,-5.62124491e-01,9.09954131e-01,7.38988042e-01,-2.29262635e-01,-1.32614136e-01,3.69171351e-01,7.58881092e-01,1.14310488e-01,-2.70877331e-02,-4.54349257e-02,2.62049228e-01,-1.13722615e-01,-4.21767980e-01,-2.89302945e-01,1.31753832e-01,-2.43629813e-01,-4.23256993e-01,-1.38922125e-01,4.89686400e-01,-2.47290060e-01,5.40434003e-01,-3.57062995e-01,2.20874846e-01,-7.21323311e-01,4.31751519e-01,3.60950202e-01,1.09633081e-01,2.61473984e-01,3.17934930e-01,-4.28825587e-01,-5.35601735e-01,2.54906267e-01,7.89006591e-01,2.13997602e-01,4.81972694e-01,-1.82425603e-01,5.23611128e-01,-4.73103762e-01,2.45224610e-01,9.01525691e-02,1.41956761e-01,-8.50615799e-02,-7.18454242e-01,1.85939416e-01,-3.52884233e-02,2.45334864e-01,3.59443724e-01,4.10603136e-01,2.84884095e-01,-1.32017031e-01,2.51439631e-01,-8.31083208e-02,1.62117854e-02,-2.80927092e-01,-1.00738394e+00,2.50923097e-01,-2.01815978e-01,1.79639965e-01,2.57311970e-01,-3.47345144e-01,-2.84040421e-01,2.47526109e-01,6.89428896e-02,-3.44557539e-02,-7.98645437e-01,-6.16998971e-01,5.89740634e-01,-4.77623105e-01,-1.53087839e-01,2.28511691e-01,-2.35052392e-01,-1.51225850e-01,2.90658213e-02,1.96320545e-02,-2.20247582e-02,3.75353813e-01,2.36227602e-01,4.63377982e-02,5.50523400e-01,2.42136329e-01,-3.68700117e-01,2.42784172e-01,-2.05451533e-01,-2.79854268e-01,-1.78204075e-01,-8.32066312e-03,1.83209762e-01,4.63183999e-01,-5.38305640e-01,3.28210648e-03,-2.85496473e-01,1.31236404e-01,-1.58277646e-01,2.55811334e-01,2.96721309e-01,1.73538044e-01,5.54208338e-01,-3.87963131e-02,-1.56782955e-01,2.18504950e-01,-4.21968311e-01,9.83840302e-02,3.52356344e-01,3.51537198e-01,-9.61633623e-02,-4.30400744e-02,1.22860089e-01,1.81436449e-01,2.51795083e-01,-3.40086639e-01,-4.33672592e-02,5.87444901e-01,1.41178325e-01,2.50345320e-01,5.06028414e-01,2.40433306e-01,-7.46609211e-01,1.62821025e-01,-5.52509785e-01,1.00990459e-01,1.38917625e-01,-3.11364710e-01,-6.54322088e-01,-5.19049391e-02,-4.04861242e-01,1.22458367e-02],[-0.3266211,-0.09011555,1.1252459,-0.14026675,-0.1427939,-0.5467645,-1.2040725,-0.04753698,-0.8173949,-0.05019059,-0.3609888,-0.19387737,-0.47603238,0.02333571,0.40791705,0.22686997,0.43661374,0.2413159,-0.04909006,-0.19179198,0.21982405,-0.17743316,-0.1573238,0.05351743,-1.3142378,-0.15788774,-0.35480383,0.34751278,0.575513,-0.2970172,-0.5002384,-0.19283308,-0.2585514,-0.29009324,-0.4947029,0.8944866,0.10320599,-0.56368774,-0.4255043,-0.17878728,-0.40974182,-0.2694015,0.19911918,0.1656519,0.543701,0.6049124,0.11633768,0.63092715,0.00982777,0.23673554,-0.05020161,-0.5099199,-0.09378791,-0.24135251,-0.1557069,-0.30158907,-0.5702108,-0.2560129,0.5355072,-0.69310564,-0.18010618,-0.38658053,-0.5043532,0.23791187,0.0315661,-0.88948196,-0.13771035,-0.44202295,0.24116904,-0.5496373,-0.4322437,-0.56222945,0.2584161,0.25388598,0.2624309,0.34437677,-1.066006,-0.67141473,-0.7061162,0.23348641,0.14221323,-0.3339848,0.09639538,0.37166256,0.20608798,0.27496916,-0.15759258,-0.51194847,0.6989664,0.0138097,0.753312,0.6240166,-0.24843849,-0.14449424,0.61843055,0.13199437,-0.02584947,-0.5086863,-0.5989827,-0.046694,-0.55250293,-0.28956488,-0.02287971,-0.13335128,-0.6801354,-0.09181668,-0.73043865,-0.62283075,-1.5278468,-0.03237097,-0.14176904,0.13699068,-0.00968072,0.20453845,0.71087,0.1567639,-0.56007123,0.01903544,0.20219287,0.6372876,0.44820014,-0.21758641,-0.46317795,-0.31068927,0.21454684,-0.33817044,-0.09405369,0.08466273,0.6362152,-0.37488917,-0.27643162,-0.09893012,-0.07497054,-0.36360198,-0.2986795,0.17806607,-1.3368684,-0.21685234,0.23709637,-0.89631003,-0.33264562,0.01230638,-0.9765677,0.05034277,-0.5196091,0.35217494,0.18457241,-0.33045715,-0.2792374,0.23484881,0.42297998,0.571346,0.43818986,0.4839881,-0.77341694,0.80314434,0.4948177,-0.012099,0.67063475,0.5800875,0.05120504,-0.02354811,-0.36241493,0.08225605,0.01347968,-0.3370588,0.7518026,-0.76158696,0.02305107,0.25924698,-0.986503,-0.10452581,-0.6465485,-0.50792176,0.76918036,-0.13417128,-0.3815823,-0.3037554,-0.08586033,0.00821781,-0.28249153,0.64377296,0.63093865,0.4672223,0.39158875,0.09350181,0.23877706,0.16939224,0.6435659,0.17330955,0.74150336,-0.20563316,-0.5481656,0.12718499,-0.09856865,-0.49119475,0.19361678,-0.4463626,-0.21303569,-0.6482529,-0.5751768,-0.22436199,-0.4367254,0.65519786,-0.28924137,-0.06900495,0.16138278,0.3573261,-0.15978082,-0.12902932,0.823216,0.01769829,-0.5748529,0.33840147,0.7734425,-0.40386724,0.24343635,-0.2940203,0.12412027,-0.26791528,0.75722426,0.03406482,0.38860962,-0.29840937,-0.4172651,-0.27382755,-0.37023103,-0.09899367,0.22225447,-0.21769208,0.70331204,-0.11820579,-0.48267266,0.22655448,-1.0363159,-0.10368922,0.2158817,0.16184081,0.19475438,0.1433116],[0.21250433,-0.515405,0.3200319,-0.48371932,-0.50081027,0.08573334,0.3545677,0.39165723,-0.01943975,0.1108324,-0.7846479,0.71813357,-0.07746689,0.31241125,0.19823992,0.17921421,0.42178565,-0.29257184,-0.35823408,-0.4346715,0.15201516,-0.2377378,-0.5087308,0.7654254,0.3323969,0.10495545,-0.57000405,0.24127437,-0.06980649,0.37211838,0.2346764,0.64963835,0.01027885,-0.13124655,-0.16925904,-0.16812102,0.03631056,0.18613043,-0.23032442,-0.18113431,-0.5397334,0.1425986,-0.53607285,0.7944502,-0.06622995,1.006578,0.4751511,0.2059798,0.3372712,-0.73421896,0.1035594,-0.53312093,0.3093483,0.42712653,0.47678974,-0.07387264,0.45928752,0.36228582,-0.19714187,0.39270827,-0.51913035,-0.13254185,0.08602949,-0.29416305,-0.4907726,0.01727837,-0.5749272,-0.5446859,0.20348899,-0.07969316,-0.46553132,-0.18966979,0.25357804,0.05335883,0.8352487,-0.2671385,-0.2515914,0.40764028,0.35832927,0.40757817,0.92284024,0.66230476,-0.04049057,0.24052624,0.41171652,0.01532415,0.27005598,0.01365991,0.08891682,0.06200942,-0.7430755,-0.5596385,-0.13296588,-0.08354943,0.04740844,-0.02761472,-0.03114766,0.53831506,1.3281536,0.21633396,-0.04109078,-0.3852735,-0.79850125,-0.50602776,0.20474547,-0.11208935,-0.09719967,0.2680328,0.20134605,-0.3366209,-0.11794537,0.50987756,-0.0697117,0.45014822,0.10096714,-0.0527768,0.5952038,0.37419775,-0.31981796,-0.00931348,0.19588539,0.16532253,1.1112018,0.23298118,-0.16054793,-0.18408568,-0.04633316,-0.157549,-1.070994,-0.49783123,0.2796273,0.05649587,0.2108268,0.26946247,0.1901845,-0.05097242,-0.5083914,0.09861777,-0.08450848,0.12905206,0.18075705,0.22745612,0.1870573,0.47860563,-0.6330908,-0.3688156,0.47404116,-0.0771917,-0.2797957,-0.22147205,0.11279875,0.02257009,-0.19676982,0.4123934,-0.83865637,-0.280191,-0.33460894,0.5713785,0.26542786,-0.22764629,0.643764,-0.02177634,-0.2061011,0.56226146,-0.05109501,0.1668676,0.00318675,-0.2741183,0.61061376,0.2390846,-0.561657,0.2438214,-0.6005984,0.50454617,-0.17818218,-0.65678173,-0.02822841,-0.14332448,0.23521428,-0.519183,-0.22854526,0.2114718,0.17112853,-0.36177304,-0.01060634,-0.87640595,0.1452171,-0.25393054,-0.0852913,-0.08964619,0.05667141,0.10314526,-0.64626414,0.2555062,0.9178208,-0.00819869,0.31408632,-0.3772536,-0.37229073,0.26073608,-0.7390493,-0.19102766,-0.09074368,0.29332432,-0.5517634,0.21585184,0.29575664,0.30468518,-0.5481584,0.1050109,-0.15333019,0.09316236,-1.1236017,-0.20218654,0.20805332,-0.26503035,-1.1406447,0.04645202,-0.52650356,-0.727021,0.23264185,-0.09145164,-0.19883934,-0.20403005,-1.0700676,-0.09936225,-0.02183347,-0.5461931,0.57823956,-1.2631421,-0.8765081,0.21000078,0.18983673,0.6892737,0.1157544,-0.32754135,-0.88766336,-0.6543681,0.16381663,-0.47882327],[-3.96024555e-01,2.59196311e-01,-7.18728423e-01,-5.66192493e-02,7.27059916e-02,-1.51846752e-01,2.71231145e-01,4.03893143e-01,2.46657077e-02,1.52557464e-02,-8.39825198e-02,-7.41773367e-01,4.46272679e-02,6.62857354e-01,-5.00722349e-01,-2.37100154e-01,1.06511511e-01,3.22923899e-01,2.11528063e-01,2.35717595e-01,1.16121545e-01,-7.01971829e-01,7.00215042e-01,-1.16462655e-01,-1.57050654e-01,1.30118817e-01,-6.68367565e-01,-3.59564632e-01,7.26888776e-02,-5.18154204e-01,-2.93238908e-01,-1.83039889e-01,1.13641441e-01,2.00644154e-02,4.33288366e-01,1.43102318e-01,5.28853893e-01,-5.75171709e-01,2.44654790e-01,1.14888020e-01,3.03425133e-01,4.62502152e-01,-1.82514787e-01,1.31149635e-01,3.76713634e-01,-9.18223113e-02,2.94691712e-01,5.00831962e-01,2.61909485e-01,3.01297307e-01,-8.01595569e-01,-6.10377118e-02,-4.70404625e-01,-2.40560681e-01,-4.25723791e-01,9.84118283e-02,2.78693020e-01,-6.60305560e-01,-2.22058296e-01,1.19937830e-01,-1.47410110e-01,2.01620594e-01,-2.35430509e-01,-5.99923432e-02,-3.31975758e-01,2.08799019e-01,2.08906576e-01,7.10134268e-01,3.36044341e-01,-5.32005765e-02,-4.68947105e-02,5.32984972e-01,8.64278898e-02,4.20933694e-01,2.31987029e-01,1.79795325e-01,4.75363374e-01,-5.58939911e-02,-2.59143472e-01,-5.30200422e-01,-2.72824436e-01,-2.75855601e-01,-2.72311956e-01,-6.29619837e-01,-1.42709129e-02,-1.85626790e-01,3.45415801e-01,-1.13644886e+00,3.85810554e-01,-6.56610906e-01,1.36007607e-01,-4.51560289e-01,-1.05741825e-02,-2.39711106e-01,4.64124680e-01,-1.34425509e+00,-5.18268764e-01,-3.40727180e-01,1.56932071e-01,7.83584356e-01,4.49722886e-01,-6.87470376e-01,1.31673411e-01,2.47518107e-01,5.88296987e-02,-2.62431562e-01,-3.88623476e-02,3.67615312e-01,4.36115593e-01,-1.12497711e+00,5.76729894e-01,-8.88656139e-01,-6.10716827e-02,-4.75609541e-01,5.75727820e-02,-5.03827810e-01,-1.07645027e-01,9.04355168e-01,-2.37676844e-01,-3.32873225e-01,-2.39681020e-01,-1.50088072e-01,3.39186043e-01,-4.31399047e-01,4.74703223e-01,-7.41870642e-01,2.77569264e-01,-7.44424999e-01,1.86539739e-01,-2.02754557e-01,-2.27708951e-01,1.09310322e-01,4.17742729e-02,4.16351445e-02,-2.34478906e-01,3.08993489e-01,-2.37131611e-01,-4.37853159e-03,4.01739746e-01,2.74421036e-01,2.73686111e-01,-3.54088247e-01,-9.12047863e-01,3.97456706e-01,-2.08622754e-01,-4.25394744e-01,-3.23449582e-01,2.16964707e-01,-4.60414857e-01,-6.26044631e-01,-2.99870700e-01,8.28229189e-01,2.57917166e-01,3.37872028e-01,-8.28222871e-01,-1.95220262e-01,-4.08583403e-01,-2.71140218e-01,1.23733133e-01,-2.40706950e-01,-3.79025996e-01,-8.65661383e-01,-9.51249778e-01,1.90804422e-01,-3.18289638e-01,8.53997990e-02,-4.27099854e-01,3.75860929e-01,-3.86278391e-01,7.95207739e-01,2.64418483e-01,8.06581080e-02,-3.18969078e-02,-4.60536391e-01,-1.46448761e-01,-2.90173203e-01,-1.04284525e+00,-7.75603473e-01,-4.41282541e-02,-3.00301611e-01,-1.26197696e-01,-6.96939707e-04,-8.88230801e-01,-2.16256246e-01,-7.92909026e-01,1.24390624e-01,-6.18877411e-01,3.52839381e-01,-3.57151121e-01,-2.95205832e-01,-7.22521096e-02,9.22516584e-02,-1.56804428e-01,-1.85430482e-01,-3.93016607e-01,4.48384494e-01,2.62019604e-01,2.51066417e-01,7.46722281e-01,2.95739442e-01,-2.65422851e-01,4.28482192e-03,-2.40205407e-01,-6.91694498e-01,3.81047204e-02,2.92058289e-01,-5.79615086e-02,1.78822383e-01,3.88256937e-01,-3.11221361e-01,3.60796124e-01,-3.98720540e-02,-3.33169758e-01,1.79967895e-01,4.60714459e-01,-1.81712046e-01,2.84969863e-02,1.32990941e-01,-2.64739394e-01,1.49981225e+00,-3.29210222e-01,-2.24927023e-01,-1.25480676e+00,4.10049409e-01,2.32015610e-01,3.67636889e-01,2.23850626e-02,-6.30702198e-01,6.84992671e-01,-5.39193332e-01,2.56953895e-01,2.51940876e-01,1.41973987e-01,-3.48484442e-02,-1.90679505e-01,1.10930875e-01,3.17481644e-02,-5.62656112e-02,-9.34603438e-02,9.45486188e-01],[3.71023625e-01,-1.94153562e-01,-3.79585952e-01,-2.46342808e-01,7.37130702e-01,-3.40476096e-01,3.52850884e-01,5.71263552e-01,4.48427387e-02,-3.45019639e-01,-5.94259560e-01,3.09571415e-01,-3.60450834e-01,6.21411443e-01,-2.10705444e-01,8.69509652e-02,-1.27836740e+00,6.93272650e-01,-2.82969266e-01,-2.70224661e-01,-2.91282743e-01,-1.58377707e-01,-7.72724897e-02,-2.87076861e-01,1.95765525e-01,-6.23642325e-01,4.35586333e-01,2.78058141e-01,4.16860998e-01,-3.24516654e-01,5.44843316e-01,-8.05696845e-01,-4.78471458e-01,-3.37721229e-01,4.21779573e-01,4.44224387e-01,-1.28493500e+00,4.65223253e-01,-1.05759643e-01,-8.07184339e-01,-3.54720592e-01,3.40203226e-01,-9.78421867e-02,5.86116388e-02,4.80509669e-01,-8.78578350e-02,3.72014076e-01,-1.42446235e-01,2.15614602e-01,-1.56523995e-02,-2.00894520e-01,-1.19186923e-01,2.17151403e-01,-2.21239910e-01,-4.19466674e-01,-5.17994344e-01,-9.65808034e-02,-2.11535573e-01,-8.61071527e-01,-3.61187011e-01,5.93707860e-01,1.97107285e-01,-2.40034088e-01,-6.69400454e-01,-9.24757719e-01,-2.85749018e-01,6.41674459e-01,2.14849770e-01,-5.83509326e-01,-2.42497638e-01,4.07592982e-01,3.93872149e-02,7.36824214e-01,-4.25796002e-01,2.17669487e-01,-5.07725358e-01,3.63759011e-01,-1.29914237e-02,7.04315142e-04,-5.91286004e-01,7.81783015e-02,9.96169031e-01,-2.14745283e-01,-1.04610153e-01,3.53351802e-01,6.57463193e-01,4.00473893e-01,-1.27340987e-01,4.77565490e-02,-2.13527650e-01,-2.66195208e-01,-4.42812711e-01,-5.14150023e-01,3.24296713e-01,2.47762620e-01,-6.03796244e-01,3.31813749e-03,-2.51069456e-01,4.82162446e-01,6.49919407e-03,3.60068053e-01,-1.84919327e-01,-2.67186373e-01,-1.84034333e-01,3.92573506e-01,3.87705177e-01,-3.05830985e-01,4.05117422e-01,-1.63503006e-01,-6.99723482e-01,-2.99903572e-01,3.60948890e-01,-2.52286285e-01,2.82101840e-01,-7.20202774e-02,1.14856474e-01,4.60364580e-01,-5.25624931e-01,-1.83432385e-01,1.98028848e-01,-6.57579184e-01,-1.76408082e-01,1.94820672e-01,9.00094360e-02,1.60920709e-01,6.15116581e-02,7.28660166e-01,1.71650887e-01,7.08552599e-02,-5.37025094e-01,-3.00129235e-01,3.39634180e-01,-1.31502092e-01,3.36075425e-01,3.66186500e-01,9.55152512e-02,1.92367777e-01,-5.92573285e-01,9.60833728e-02,2.62938499e-01,-3.47988635e-01,-5.64865060e-02,6.06298149e-02,-5.81754684e-01,-5.83209276e-01,2.88239121e-01,-1.44606650e-01,7.73277521e-01,1.41927242e-01,-2.32604370e-01,-7.64511749e-02,3.93687636e-01,4.72163647e-01,1.58044472e-01,1.61955401e-01,-2.42254347e-01,-5.83265662e-01,6.23993754e-01,-7.10012987e-02,-4.03961278e-02,-1.19946539e-01,-1.04709566e-01,-5.05259931e-01,1.72795907e-01,-4.11663234e-01,-5.97471893e-01,-2.49723077e-01,-2.45955572e-01,3.00612509e-01,-2.42476910e-01,1.54047236e-01,3.87018740e-01,-4.46516782e-01,3.04342002e-01,5.54375350e-01,-2.78702170e-01,1.35797903e-01,-2.74695996e-02,-2.64623135e-01,3.74859333e-01,-4.17231023e-01,1.57318488e-01,2.89313048e-01,3.18005264e-01,-1.30292922e-01,-5.50349116e-01,-4.49550956e-01,-3.41345459e-01,-5.08195043e-01,-8.43031585e-01,3.57644439e-01,-6.31194264e-02,-2.52658010e-01,5.32959700e-02,-7.07605660e-01,9.74730372e-01,4.30431932e-01,-3.18161190e-01,-6.65334463e-01,-1.68935642e-01,4.64347638e-02,-3.60945553e-01,9.05502662e-02,4.93919343e-01,1.49700254e-01,-5.93910515e-01,-9.54487860e-01,-1.61157891e-01,8.98107708e-01,-2.75555432e-01,1.03067195e+00,2.07826123e-01,2.38312498e-01,-9.66695547e-02,3.25110674e-01,3.29118252e-01,-2.26398513e-01,-1.48322552e-01,1.66932553e-01,-8.82151797e-02,2.75906175e-01,-8.74995589e-02,-1.28253281e+00,6.75893724e-01,4.29767400e-01,-3.54640968e-02,3.67696464e-01,4.38767880e-01,-1.71676785e-01,3.88710529e-01,-6.07959390e-01,4.84698772e-01,6.89531863e-03,-2.99116522e-01,-2.99697191e-01,4.18839157e-01,2.19053075e-01,-7.58373588e-02,-3.64020884e-01,-1.61944494e-01],[-0.09317309,-0.49375585,-0.4670161,-0.23790453,0.06291261,-0.0844281,-0.2634452,0.17229381,0.04222568,0.15407512,-0.07664838,-1.1123271,-0.76958686,0.23864938,-0.40109715,0.34930536,-0.08481162,0.58886945,0.2815087,-0.14533132,-0.1252102,0.55378467,0.1575135,-0.47219586,-0.25580546,0.22855264,-0.15455878,0.0905106,-1.1768347,-0.15931274,0.16906306,0.10505898,-0.24753934,0.8169736,0.31491506,0.42674232,0.02650031,-1.8338498,0.06543076,0.01963084,-0.1739145,0.02015003,-0.03577068,0.36000833,0.1976625,0.21391359,0.28015974,-0.5820477,-0.1245154,1.0899142,-0.10965931,-1.4696426,-0.07716511,0.6762602,0.4132694,-0.2649267,-0.8566755,0.08625072,1.3365008,-0.68285996,-0.2960006,0.02714172,-0.4255405,-0.15366273,0.02337889,0.23903282,-0.08619858,0.3541048,-0.14755501,0.10850886,0.09699376,0.7480479,0.3570119,-0.6556195,0.8412339,0.30327034,-0.12136031,0.14824882,-0.4843455,-0.20434204,-0.10990562,0.00629858,-0.23145558,-0.21982513,-0.8583536,0.33566636,0.19792646,-0.6721231,0.44310445,-0.53245413,0.08937426,0.37447226,0.24803908,0.35712594,-0.23084193,0.17198275,0.03129443,0.8168508,0.11483431,0.48241398,0.6649945,-0.31770673,-0.38789836,0.02602241,0.3230504,-0.8500078,-0.02462611,-0.518742,0.11248968,0.26378328,-0.6412673,-0.11517415,-0.03165371,-0.00365496,-0.07646876,0.2271371,-0.57896835,0.00375515,0.36695835,0.2756522,-0.04483114,-0.9191924,-0.05107884,-0.5185796,0.567559,-0.4507231,0.2541576,-0.14796036,-0.97448045,0.20863971,-0.10971554,0.27887416,-0.2694764,-0.10824435,-0.15155457,-0.1673155,0.50330293,0.4780754,0.52018005,-0.10231015,-0.30767378,-1.1169716,-0.09965613,-0.50171465,0.10220794,-0.41884887,-0.39325738,-0.19116081,-0.39833528,0.05583264,-0.43828905,-0.19962704,0.20183645,-0.20741943,-0.22171211,-0.01110863,-1.0955397,-0.97527647,0.26218545,-0.48915896,-0.01212505,-0.37080786,-0.48499772,-0.34370354,-0.66123754,0.10577899,-0.2100207,-0.62203217,0.51447797,0.58336145,0.72587025,0.16203992,-0.44295728,-0.5045455,0.5115594,-0.23825365,0.15595461,-1.0420544,-0.0243587,0.12143244,0.15686178,-0.02405469,0.11263721,0.30801937,0.5236189,-0.28133506,-0.40855247,-0.11647175,0.2731825,-0.6164218,-0.28297105,0.8170469,-0.38982433,0.21879727,-0.21462803,0.01478173,-0.21453933,0.20436257,0.26936862,-0.4372309,0.18725961,0.19443265,0.60946274,0.35145995,0.01338963,0.6161069,-0.00543018,0.19284792,0.30273715,-0.16337414,-0.14286624,-0.4601387,0.0435883,0.5969653,-0.5054456,-0.11611884,0.17950967,-0.77357656,-0.07682589,-0.28790718,-0.08359545,0.41298506,0.37580705,-0.22321196,0.5847125,-0.04975058,-0.60073113,0.7613673,0.26789185,0.43399486,-0.48272622,-0.3095932,0.18260832,-0.14060281,-0.2770028,0.0633137,-0.4704255,0.01436645,0.32549664,0.04080125],[-5.5328965e-01,-1.0357754e+00,4.1322657e-01,1.5152560e-01,-7.7325368e-01,1.3748603e-01,2.0331882e-01,2.5875416e-01,8.0727033e-02,-1.5942270e-02,2.4442324e-01,-3.7741917e-01,2.0085879e-01,-5.8116484e-03,6.9555318e-01,3.1339377e-01,-4.4046620e-01,3.1385757e-02,4.3955889e-02,1.0498048e-01,1.9006124e-01,-3.4972269e-02,6.1261541e-01,-6.8583471e-01,1.9219555e-02,4.6835628e-01,1.5569867e-01,-2.4672320e-01,3.0348974e-01,-4.1676828e-01,4.1055933e-02,-7.0756489e-01,2.4309608e-01,-1.0045990e+00,-3.7542764e-02,-3.9189544e-01,-4.9346637e-02,1.9808136e-01,2.8523824e-01,-1.3390483e-01,-2.7149460e-01,-3.8208047e-01,-1.6160177e-02,4.2130941e-01,-1.0213137e-01,4.4543602e-02,-9.1449630e-01,2.5347376e-01,1.4313634e-01,-4.2608249e-01,-2.8715885e-01,-2.0922130e-01,-3.6688390e-01,2.7622947e-01,1.5777923e-01,-2.0071673e-01,4.1089830e-01,-1.7932937e-01,-2.8453314e-01,-1.5477742e-01,-2.4279809e-01,-2.2824447e-01,-3.4756526e-02,-4.2347625e-01,2.4268287e-01,2.7664262e-01,3.3461171e-01,-3.4759137e-01,-4.1684574e-01,-8.7246221e-01,1.0511701e+00,-5.1189137e-01,1.0494436e-01,-1.8773450e-01,-1.4913803e-01,5.1831488e-02,1.7683672e-02,-2.3523870e-01,-5.3195941e-01,3.2636699e-01,4.2719608e-03,-1.3258244e-01,2.0736593e-01,-1.1190190e-01,-1.2496849e-01,1.6951992e-01,-4.0991921e-02,1.8297802e-01,5.3428763e-01,-1.4170681e-01,-1.3718933e-01,2.8511560e-01,7.7376947e-02,-2.4702668e-01,1.2687592e-01,-2.6445428e-01,6.8800491e-01,-2.5906387e-01,5.4491264e-01,-7.8541443e-02,-5.3811550e-01,4.0619358e-01,-2.9852781e-01,-4.1985312e-01,5.4696810e-02,2.1635547e-01,-1.2924674e-01,1.1937390e-02,5.1643705e-01,1.3583273e-01,2.5180222e-02,-5.0269520e-01,-1.5657471e-01,-2.2180957e-01,-4.4208533e-01,7.7561510e-01,1.5644161e-01,-6.3984357e-02,3.3730841e-01,4.8793951e-01,-6.9225594e-02,-2.3752096e-01,3.2681215e-01,-8.2347858e-01,-6.9312207e-02,6.1082667e-01,-6.2898241e-02,-7.8504488e-02,-7.8155130e-02,2.6698247e-01,-7.1727008e-01,-6.5599018e-01,1.4599563e-01,4.4723675e-01,-4.1321689e-01,1.8079855e-01,-2.4842450e-01,-1.4656584e-04,-5.8659124e-01,4.9441317e-01,1.7501555e-02,4.3952871e-02,-6.3772693e-02,5.8001395e-02,2.8209430e-01,1.9757544e-01,2.8269267e-01,-1.4439958e-01,1.6213657e-01,3.7466857e-01,1.2631404e-02,9.8885171e-02,9.2222601e-01,-6.0934991e-01,6.6564754e-02,-1.8358203e-02,-1.3067648e-01,-3.8866568e-02,5.4683262e-01,6.8614669e-02,8.0253822e-01,-3.7855729e-01,-5.9933978e-01,3.6780101e-01,-2.1128331e-01,-5.6484681e-01,4.8115829e-01,-1.7526907e-01,4.2001638e-01,-5.6781733e-01,-6.1425006e-01,-1.6457184e-01,-8.6369562e-01,-5.5315888e-01,-7.2820596e-02,-3.5060951e-01,-2.1906021e-01,9.7878581e-01,-2.8358585e-01,-1.5859644e-01,-1.9605441e-01,-3.7975284e-01,2.8016186e-01,-1.7587312e-01,-1.1770256e-02,-5.0775174e-02,2.0068915e-01,-6.4638004e-02,-3.4849423e-01,1.2596202e-01,-1.1259667e-01,-2.6617402e-01,3.0646867e-01,-5.7650143e-01,-5.3066140e-01,-1.2337795e+00,-3.9970887e-01,2.8772122e-01,5.2622741e-01,-4.3492740e-01,3.3101134e-02,-4.5805056e-02,4.8085594e-01,6.3383889e-01,1.4305905e-01,-9.5774937e-01,5.1022023e-01,-1.4384384e-01,5.8369316e-02,-2.2528338e-01,-2.0183980e-01,6.9396931e-01,-1.7218368e-01,7.3952901e-01,-1.9988455e-02,1.6442707e-01,-8.4038490e-01,1.1197746e-01,8.0379635e-02,5.6309503e-02,-1.8235074e-01,1.0068527e-01,-2.4857838e-02,-7.3370111e-01,6.0194328e-02,-7.7197015e-02,-3.0013672e-01,-3.6818400e-02,1.2762363e+00,-8.1811301e-02,-1.2758222e-01,2.0316054e-01,-1.9718850e-01,5.7317388e-01,-3.2211408e-01,3.9455888e-01,-2.4434027e-01,-2.2938766e-01,5.0036412e-01,1.7175290e-01],[6.10438645e-01,4.02464598e-01,1.78682894e-01,-3.67885113e-01,4.38970653e-03,-2.59927362e-01,-2.53507018e-01,5.16733944e-01,-1.82078645e-01,8.15511644e-02,-5.82604587e-01,-1.38743901e+00,-2.66498655e-01,1.31764218e-01,-9.00952553e-04,7.61201780e-04,-6.42131925e-01,1.15886480e-02,-5.74105740e-01,-4.72497083e-02,1.31692559e-01,2.34469533e-01,-6.52382672e-01,-8.03836644e-01,4.98530805e-01,-1.63464859e-01,-3.04246366e-01,4.82360482e-01,7.86849201e-01,4.52671319e-01,-2.40239173e-01,1.15662672e-01,-7.95048177e-01,-3.76544535e-01,-5.31832613e-02,2.80110538e-02,5.18351197e-01,-5.81018984e-01,-5.63592184e-03,-2.47450843e-01,-5.54320775e-02,-4.32311416e-01,6.13936126e-01,-1.60388798e-01,-5.97621977e-01,9.47948825e-03,1.99287117e-01,9.85244811e-01,2.23500088e-01,-4.93117183e-01,5.96840739e-01,-5.84451258e-01,1.10609621e-01,-1.14265457e-01,-7.98695624e-01,-6.75496683e-02,6.03837192e-01,2.14277860e-02,3.57687712e-01,5.29473126e-01,4.49767569e-03,5.40367961e-01,2.53932506e-01,2.87956446e-01,-1.14833617e+00,-1.64009467e-01,1.22082627e+00,1.37529522e-01,1.24070503e-01,-2.44848698e-01,3.84290144e-02,-5.58913648e-01,-1.02870166e+00,7.53668547e-01,-3.89560074e-01,-3.31434250e-01,-6.06980562e-01,3.23471397e-01,-3.02240163e-01,5.62650502e-01,-1.61530092e-01,-9.62328672e-01,1.79949805e-01,2.29158297e-01,-2.34259278e-01,6.61852717e-01,-6.72535658e-01,3.91543917e-02,4.61959749e-01,-5.93079150e-01,-4.79066253e-01,-1.24405906e-01,-4.77740867e-03,6.45207644e-01,9.55056176e-02,3.34618636e-03,-1.00917995e-01,-1.18421078e+00,1.36316448e-01,3.68549258e-01,-5.13766482e-02,-9.19013098e-03,-4.92097199e-01,-5.35048902e-01,1.91535592e-01,3.57152522e-01,6.27785847e-02,2.19732081e-03,4.41655248e-01,-5.60269970e-03,1.98031336e-01,3.07761937e-01,-6.44984841e-01,-8.68055373e-02,-2.89041191e-01,-2.96698540e-01,5.59324145e-01,1.16106316e-01,1.52872011e-01,-2.32836545e-01,-9.35731754e-02,4.43341672e-01,8.44613194e-01,-3.65765154e-01,5.16544402e-01,2.71477312e-01,1.00466967e-01,7.28835523e-01,4.36898728e-04,8.39552045e-01,-2.95066804e-01,3.76210064e-01,-8.47177953e-02,-5.73906787e-02,3.20144892e-01,4.00404871e-01,2.03870237e-01,-3.72637957e-01,9.89508629e-02,1.20807327e-01,7.75892884e-02,-1.59289405e-01,1.02877907e-01,4.89780486e-01,-8.30406070e-01,-5.22536226e-02,-1.35639636e-02,3.28529440e-03,3.18223298e-01,1.72836095e-01,-8.29774499e-01,4.72346283e-02,5.82054816e-02,-1.19617179e-01,-2.29795337e-01,-5.18450677e-01,-9.40742418e-02,-4.51410770e-01,-5.64609945e-01,3.53354722e-01,-6.26555681e-02,9.72889811e-02,4.94077533e-01,4.45473075e-01,8.73038620e-02,2.30596568e-02,6.55583560e-01,-1.42450348e-01,-2.28264779e-02,3.12863514e-02,-7.79213235e-02,-1.51188940e-01,6.84537888e-01,3.39037240e-01,-9.31168720e-02,-3.02111030e-01,7.10977316e-02,-4.30481359e-02,-2.91583985e-01,-1.33578882e-01,2.08561733e-01,8.52707252e-02,-4.33613896e-01,4.05364931e-01,2.86911994e-01,-1.56892791e-01,2.00427040e-01,5.42002976e-01,1.19687945e-01,-2.30829433e-01,2.98558008e-02,-1.14325368e+00,3.46135855e-01,-1.65616766e-01,-3.84293973e-01,9.12676230e-02,1.37739286e-01,1.79511473e-01,1.12228431e-01,4.86898385e-02,-3.27253044e-01,1.15642942e-01,3.42068642e-01,4.05468225e-01,1.04613557e-01,7.92442739e-01,4.32888448e-01,-7.02848017e-01,-3.83990049e-01,5.77292383e-01,-4.68790352e-01,6.54621869e-02,5.58281302e-01,-6.08586013e-01,-5.84615648e-01,-4.19322193e-01,-1.21179707e-01,1.14151560e-01,2.00135291e-01,-5.65755785e-01,7.97101200e-01,-5.66388369e-02,-3.83253336e-01,2.27462202e-01,5.39919674e-01,2.86802709e-01,-2.64105815e-02,6.97123725e-03,3.88075888e-01,-1.33204743e-01,-5.47767997e-01,-2.59769440e-01,-5.85471652e-02,-3.30032796e-01,-8.54968578e-02,8.17119896e-01,-4.60430562e-01,-8.92786086e-02,-2.40323111e-01,2.43954509e-01],[-1.59512348e-02,-2.71091592e-02,-1.22488379e-01,-1.06202853e+00,-2.69330055e-01,2.48358607e-01,3.75195704e-02,3.44351709e-01,-1.86669737e-01,-8.05101037e-01,3.45738322e-01,-1.99275598e-01,-8.83221984e-01,1.08205724e+00,-8.69648680e-02,-7.32969761e-01,-5.57988167e-01,2.86196500e-01,1.03969701e-01,-5.52307427e-01,-1.71197504e-01,9.76290703e-01,-3.19585621e-01,1.35435820e+00,-4.05815631e-01,4.03698981e-01,-3.43118101e-01,-1.51282644e+00,-4.54177916e-01,2.66560644e-01,2.50558078e-01,1.00094646e-01,1.00947462e-01,-1.00855902e-01,2.32782543e-01,9.15588513e-02,3.95067185e-01,1.98762298e-01,1.88276529e-01,-6.11834347e-01,-4.47655730e-02,-3.47976029e-01,-1.12739593e-01,-2.79231310e-01,-3.89715642e-01,3.94840151e-01,-4.92007881e-02,7.37978339e-01,-6.71736598e-02,-1.89275160e-01,3.42356414e-01,2.65395194e-01,2.24036574e-01,-8.06190372e-01,2.61219859e-01,1.54465079e-01,-2.69675534e-02,2.39385486e-01,3.55358154e-01,4.00312871e-01,-4.27206814e-01,5.72377324e-01,1.58285588e-01,2.11552382e-01,-3.41724418e-02,1.11059546e-01,-1.31936735e-02,-5.66327989e-01,8.82259756e-02,1.48038149e-01,2.92366985e-02,-5.59375696e-02,-4.84572917e-01,1.35442585e-01,1.08483583e-01,8.38940311e-03,5.11635654e-03,-5.56217432e-02,-4.97565746e-01,-1.11282563e+00,1.38454407e-01,-5.89774609e-01,-4.54690084e-02,-2.25291461e-01,-2.33403012e-01,5.06537259e-01,1.00208797e-01,9.98198427e-03,-2.68884659e-01,-8.48228991e-01,-1.82012729e-02,-6.06435239e-02,-8.49415779e-01,3.53354394e-01,-1.47558779e-01,8.09767544e-01,-1.03844449e-01,6.21037006e-01,1.41377017e-01,3.28429103e-01,8.11291158e-01,1.88687131e-01,-2.64720738e-01,-8.23731065e-01,-3.93984079e-01,1.92164361e-01,1.21866785e-01,3.58463436e-01,3.52318138e-02,1.79158479e-01,3.53880912e-01,-1.09843862e+00,-2.10421681e-02,-9.93294939e-02,1.43955335e-01,3.42762887e-01,5.84713340e-01,-8.45025182e-01,-9.11537558e-03,3.95469546e-01,5.88474981e-02,-4.75918442e-01,3.51573735e-01,4.15065020e-01,4.14889455e-02,4.99582767e-01,-5.34641789e-03,-5.93332946e-01,-6.49466038e-01,-2.64981031e-01,5.51115647e-02,6.20048463e-01,-4.43159491e-01,-1.02391548e-01,3.32729191e-01,-9.50813852e-03,-6.57538176e-02,-5.38067877e-01,4.39823009e-02,5.87202549e-01,-4.27504420e-01,4.26249981e-01,2.09059969e-01,-7.30915308e-01,-1.38595670e-01,4.01595622e-01,-2.21588224e-01,-3.91391754e-01,5.50408754e-03,-7.25253582e-01,1.24648042e-01,-3.56746137e-01,-7.49362886e-01,7.62942135e-02,2.68273562e-01,-4.96832222e-01,-7.12872326e-01,-9.87637579e-01,-4.34775501e-02,-3.68805826e-01,1.13224760e-01,6.56695306e-01,2.76208013e-01,-4.62832928e-01,-2.55806804e-01,-9.61680934e-02,-3.99677962e-01,-1.55441165e-01,-7.90075719e-01,-6.36268854e-02,7.52732828e-02,-2.64817208e-01,3.43057871e-01,-1.42962515e-01,3.88081461e-01,3.12700272e-01,7.56539166e-01,-3.60725403e-01,-6.83306903e-02,3.66338193e-01,2.36212254e-01,-1.22600235e-01,-2.30879709e-02,-1.15352392e+00,-3.14268976e-01,-8.34010661e-01,-1.41024932e-01,-2.72645861e-01,3.19412470e-01,-7.37363935e-01,-1.65973797e-01,1.00124645e+00,-1.94869116e-02,-4.18061018e-02,-2.49868557e-01,-1.65063465e+00,-2.73664277e-02,1.23073170e-02,-3.97130698e-01,2.04451218e-01,4.08376232e-02,-3.15638900e-01,6.54180720e-02,5.83100617e-02,3.11949044e-01,-2.27178156e-01,4.08321649e-01,1.10063180e-01,9.98814497e-03,4.63885635e-01,-3.22305471e-01,3.32673609e-01,-1.75688952e-01,-2.21460164e-01,2.26590350e-01,-1.13444829e+00,-3.01214486e-01,-5.33967018e-01,2.39087120e-01,-9.73695815e-01,-3.49751323e-01,4.94988710e-01,2.85783023e-01,-3.98260772e-01,1.41201885e-02,-2.91389559e-04,-1.50369614e-01,3.86976302e-01,-2.19205648e-01,2.75342047e-01,-1.25301806e-02,-5.41935682e-01,-1.63490236e-01,-1.53200582e-01,2.66658694e-01,-1.19900711e-01,-3.91711220e-02,1.88473731e-01,-3.88685316e-01,3.05322707e-01],[-0.44028333,0.1531324,-0.33208236,-0.6327353,-0.8914378,-0.62022215,0.18469365,-0.2440925,0.0601869,-0.770437,-0.79985845,-0.44863433,0.6339732,-0.20483166,0.00910902,0.2716813,0.33790812,0.7861711,0.53327596,0.01067443,-0.18307392,0.31451017,-0.26175022,0.20042925,-0.34866172,0.10811052,0.6223256,-0.10561527,0.23282942,-0.64081204,0.23543629,-0.34907472,-0.05325245,-0.1435981,0.09070391,0.34000638,-1.1153653,0.0624423,0.18461365,-0.42694542,-0.01279872,-0.14180662,0.34130713,-0.10590407,-0.11775958,-1.0948275,0.02231204,0.04112493,0.51373816,-0.47868764,-0.469298,-0.19369909,-0.18501097,-0.21930474,0.6024155,0.5288968,-0.21216054,0.10522979,0.40586925,0.4122008,-0.02934696,0.93333185,0.19968946,0.16407238,-0.5821128,-0.08954375,0.49090698,-0.16337574,-0.02905771,0.3326699,-0.20099689,-0.39796805,0.08467662,-0.40111277,0.01475616,-0.8304682,0.48560554,-0.5208373,0.4282489,-0.18560182,0.05856222,0.45575967,-1.1161842,0.55464864,-0.17227873,-0.21090746,0.34233645,-0.09550495,0.35337454,0.19819035,-0.6868508,-0.17874405,-0.13406648,0.01217898,0.16689977,-0.24482825,-0.0637083,-0.15117267,0.10753171,0.7955175,-0.07638738,-0.13629308,-0.1340503,-0.22527492,-0.19718339,0.45920563,0.1674598,-0.2552082,-0.05986932,0.4527392,-1.1788,-0.6499933,-0.45195758,0.56407434,-0.6017854,0.45126572,-0.8384322,0.32872224,-0.31142038,0.18460938,-0.30233955,-1.2573423,0.53708184,-0.07757791,0.19660932,-0.3040603,-0.13833229,0.20943002,-0.17094377,0.05747948,-0.29337096,-0.18544231,-0.6844605,-0.88984215,-0.066102,-0.31372195,-0.23751315,0.26747942,0.1817556,0.4570386,0.04932737,0.4803429,0.96002525,-0.09749423,0.43077162,0.79873055,-0.49851817,-0.12912288,-0.00251155,-0.47779998,0.27333462,0.5825561,0.06522701,-0.3443964,-0.75879425,-0.72350043,0.01224939,0.09572461,0.01670886,-0.06408578,0.11131188,-0.30561242,-0.2447181,-0.03696182,0.2888303,-0.18929543,-0.06155967,-0.18736857,0.4205628,-0.4437936,-0.25255015,-0.09824986,-0.8026614,-0.34925154,0.06917802,-0.7826506,0.06393588,0.284136,0.05156401,-0.08350111,-0.60979927,-0.6247058,-0.4026325,0.69915485,0.34547585,0.3195807,0.05388608,0.5157765,0.30165583,-0.01104851,0.5222792,0.3094746,-0.36714593,0.05397646,0.15630777,-0.5810563,-0.82917374,0.02694928,0.08381852,0.07771347,-0.1285881,0.48192376,-0.28522193,0.4582935,-0.7480452,0.15390497,0.6626149,-0.16758132,-0.14933747,1.3236613,0.05025128,0.26177782,0.697135,0.23699877,-0.66877025,-0.6565281,-0.3419651,0.16483544,-1.1967599,0.64164424,-0.6394497,0.0974874,0.40465173,-1.0111557,-0.75010246,-0.1027609,-0.01090298,-0.05421638,0.16090372,0.8561493,0.25083905,-0.35349086,-0.18342204,0.33416152,0.12221055,-0.5658435,0.29850683,0.55223536,0.13365673,0.43091533],[-4.57609028e-01,1.46571510e-02,-2.02009484e-01,1.94483232e-02,-4.03629929e-01,-1.13037135e-03,-3.95762861e-01,-4.94711883e-02,-3.64165246e-01,8.08156431e-02,-2.97295660e-01,-3.26432914e-01,-2.02627897e-01,8.39266032e-02,1.33404329e-01,-7.89589286e-01,-7.17837214e-02,4.30555820e-01,4.24552500e-01,-2.44552433e-01,-3.47348660e-01,-2.22063482e-01,-2.23010883e-01,2.22125843e-01,-2.13794425e-01,-4.37410325e-01,4.62561667e-01,-1.76620856e-01,-5.22585571e-01,-7.94671476e-01,6.34505153e-01,3.62065583e-01,-1.19542763e-01,2.35110939e-01,-7.20105171e-01,2.61161834e-01,-6.05122089e-01,-5.55225909e-01,-2.33147368e-01,-2.71876872e-01,2.63207197e-01,-1.62293732e-01,-1.10071130e-01,1.92955285e-01,-1.40957758e-01,2.71363109e-01,-2.21886724e-01,-4.87510502e-01,1.61857888e-01,-2.33968288e-01,2.71399677e-01,2.04877153e-01,-3.31605464e-01,-7.43343711e-01,4.16080236e-01,1.23323418e-01,-1.04683352e+00,-6.03724420e-01,-2.65738696e-01,2.35600621e-01,2.55627781e-01,1.56035796e-01,-5.11749201e-02,-4.30065483e-01,-4.64669943e-01,-1.57834664e-01,-6.89123929e-01,-1.07590163e+00,-2.02322677e-01,7.43233860e-01,-3.67363006e-01,-1.27933145e-01,-7.52914310e-01,3.59293193e-01,2.65161574e-01,-3.41185600e-01,-2.68097706e-02,-1.16459228e-01,-1.19822550e+00,1.65201306e-01,-1.99969187e-01,3.03909034e-01,-3.20152551e-01,1.79355726e-01,-9.60344553e-01,-6.64813697e-01,4.16647106e-01,-3.51195753e-01,-2.78608739e-01,3.70991111e-01,-1.64538786e-01,-7.33297527e-01,9.91885364e-02,-4.76048708e-01,1.30365416e-01,-5.91361642e-01,2.43122846e-01,2.33164743e-01,-1.24451540e-01,5.07916845e-02,-4.13871795e-01,-3.72699648e-01,-2.73923755e-01,-3.61298680e-01,-8.42943788e-02,-4.50159788e-01,-2.81046718e-01,3.81651968e-01,2.73683280e-01,-3.22176963e-02,-3.47074568e-01,-9.12825644e-01,-3.06110710e-01,-5.71575582e-01,-1.14086235e+00,-2.92236269e-01,7.21082866e-01,-2.45802417e-01,-2.31347322e-01,-1.36936624e-02,4.06521678e-01,3.21825266e-01,1.91943124e-01,5.59144437e-01,1.05842687e-01,2.04599068e-01,8.40725899e-02,3.21031690e-01,5.41063964e-01,1.84355360e-02,-2.19952151e-01,3.19929183e-01,-5.62286489e-02,-1.71370074e-01,-1.08732367e-02,9.53079313e-02,-6.16869926e-01,4.94322509e-01,3.15252364e-01,-3.29833597e-01,-1.06072076e-01,8.12450707e-01,4.93391484e-01,-3.87451768e-01,5.91053903e-01,4.96729799e-02,2.26556823e-01,4.69061017e-01,4.05039668e-01,4.98069644e-01,-5.13901889e-01,2.26962745e-01,1.30325109e-01,2.97789246e-01,-9.48843598e-01,-2.53991425e-01,6.21302389e-02,1.63931429e-01,-2.15320930e-01,4.97069329e-01,-7.76561722e-02,9.61195230e-02,-2.22840667e-01,-9.94274080e-01,-1.94111258e-01,-4.40333821e-02,-1.33120090e-01,5.20614028e-01,5.40815532e-01,-2.22300798e-01,2.80672252e-01,2.24336341e-01,-2.73187831e-02,-9.46613848e-01,8.90459478e-01,2.21790820e-01,5.40756643e-01,-9.33490843e-02,-3.41737777e-01,3.25855762e-01,4.13819313e-01,1.36301592e-01,-8.98201987e-02,-1.76949173e-01,-1.63326234e-01,-3.81734176e-03,-4.47950922e-02,2.79918462e-01,-6.71894193e-01,1.09253383e+00,-6.33054197e-01,-1.27733245e-01,-1.77335992e-01,1.31917059e-01,-3.81540865e-01,8.66788983e-01,3.10020536e-01,-1.60626441e-01,3.61964941e-01,-2.38376275e-01,9.14941579e-02,1.44873895e-02,3.08025837e-01,3.82372141e-01,-1.82867110e-01,-7.23839924e-02,-6.48535132e-01,2.56953090e-01,1.81939065e-01,5.04559159e-01,-1.35139942e-01,-3.27340305e-01,1.81133181e-01,3.00667882e-02,5.39888799e-01,-1.01154959e+00,8.93515468e-01,2.06982732e-01,3.32566142e-01,-5.10914803e-01,8.42307806e-01,1.13726467e-01,8.06719363e-01,-2.07101196e-01,6.76206052e-01,2.53526360e-01,1.30655477e-02,-7.02584267e-01,-2.20190808e-01,3.07499677e-01,-3.54501814e-01,4.98569012e-02,2.78830022e-01,-1.16052818e+00,-9.36098322e-02,8.07498023e-02,1.89005926e-01,2.12524962e-02,-7.11896241e-01,1.74854010e-01],[-7.35725090e-03,-1.36706293e-01,-2.18312591e-01,-7.69735947e-02,-3.19848686e-01,8.98287892e-02,8.67669433e-02,2.67575592e-01,2.52447784e-01,5.52413583e-01,1.76265076e-01,-7.09134266e-02,-6.23986602e-01,-2.52690852e-01,-1.99429274e-01,-4.19198662e-01,1.54977053e-01,-2.27735519e-01,-4.71911788e-01,4.77078110e-01,4.77611870e-01,-7.89404094e-01,3.83610964e-01,-6.54268622e-01,4.68265146e-01,2.86990821e-01,2.33229116e-01,-1.21513462e+00,1.44936770e-01,-3.41448098e-01,-3.25178236e-01,1.83668301e-01,1.53521299e-01,-3.24809998e-01,2.76617527e-01,8.72457922e-02,-8.06811392e-01,1.50465116e-01,-5.21663725e-01,1.56068131e-01,-2.29206160e-02,7.58963227e-01,-1.42117232e-01,-1.18995421e-01,-1.93768188e-01,-4.63763595e-01,4.79641825e-01,6.19840585e-02,-5.85181713e-01,-8.44159186e-01,2.58702964e-01,1.42837450e-01,1.77690744e-01,3.57422531e-02,-4.02800471e-01,-2.23398879e-01,-3.82502228e-01,-3.82515311e-01,-1.21094242e-01,-4.01892334e-01,5.37156582e-01,-4.59603280e-01,-9.90428962e-03,1.49970293e-01,2.06195414e-01,-2.03339681e-01,3.42420369e-01,-3.19143444e-01,-1.36397868e-01,7.47785047e-02,-1.89132139e-01,-4.74413246e-01,1.76568314e-01,1.14439487e-01,7.05726445e-01,-5.92211604e-01,1.46874040e-01,8.47376049e-01,1.94610447e-01,-6.99117959e-01,-2.10282728e-01,1.39710709e-01,7.32816935e-01,-2.20836043e-01,5.17563462e-01,2.79719621e-01,-5.59566617e-01,-1.71594754e-01,-6.44034624e-01,1.34394050e-01,-6.43933952e-01,3.74468714e-01,3.41093428e-02,5.89594245e-01,-2.20400155e-01,-6.99346602e-01,-8.76095742e-02,-2.13658616e-01,-1.73341185e-01,7.69115090e-02,-8.41327190e-01,-6.52621031e-01,-5.19965589e-01,4.91343170e-01,-6.22995913e-01,-6.01943195e-01,2.15444118e-01,3.79022866e-01,2.11967632e-01,-2.78288156e-01,5.08711517e-01,5.34637392e-01,4.28880788e-02,6.09026968e-01,2.52628386e-01,-4.96508300e-01,-2.00640187e-02,5.02354503e-01,-2.60941178e-01,4.96292561e-02,1.73786581e-01,-1.42308208e-03,2.14791879e-01,1.72252338e-02,-4.19069767e-01,4.44177240e-01,-1.42311609e+00,1.56313658e-01,6.98575854e-01,1.31497547e-01,7.44684935e-01,-2.75103338e-02,3.85902613e-01,1.16388731e-01,-1.92095280e-01,1.01074338e-01,-1.38031051e-01,-3.89294624e-01,7.74370968e-01,-6.55016124e-01,-6.76528439e-02,6.32366240e-01,-1.88477505e-02,3.97186905e-01,2.92518407e-01,4.48734313e-01,4.47642654e-01,3.66326869e-02,-4.25132830e-03,-3.52344215e-01,-3.53649765e-01,1.77473560e-01,-1.60100043e+00,1.68415770e-01,3.45518798e-01,-1.35448292e-01,7.25869894e-01,3.87517422e-01,-4.37644511e-01,7.41906106e-01,9.80836339e-04,-5.00967741e-01,-3.13305795e-01,3.08500558e-01,-3.74217778e-01,-2.69014180e-01,1.62147969e-01,-3.64515156e-01,-8.73583928e-03,1.61701500e-01,-3.19310188e-01,3.76943082e-01,6.93008482e-01,-6.10650897e-01,5.77291697e-02,-3.03285569e-01,-9.87097085e-01,-3.05840909e-01,3.49376768e-01,3.01127762e-01,4.45192367e-01,-5.04104376e-01,4.97715056e-01,-4.94265705e-01,-6.22131050e-01,1.23603791e-01,-3.99879932e-01,3.78446698e-01,1.52683511e-01,-6.08363867e-01,2.05497742e-01,-3.40546608e-01,8.47251773e-01,-1.41607061e-01,1.38044357e-01,3.35781991e-01,-5.13601840e-01,6.29707336e-01,-1.44216925e-01,-3.20119858e-01,3.50724012e-01,-1.42658925e+00,-5.87365389e-01,-4.71189320e-01,-3.82240787e-02,7.92418957e-01,-9.90724191e-02,-1.75471634e-01,5.06521583e-01,-1.18441723e-01,3.64724100e-02,-1.00066459e+00,-3.65188420e-01,-6.38887882e-01,2.95097768e-01,-6.31064057e-01,3.36590439e-01,8.24540555e-02,-3.34918089e-02,-3.70748937e-01,-1.43348277e-01,1.79746792e-01,-9.55172926e-02,8.61413062e-01,1.00451544e-01,6.61678493e-01,8.14189538e-02,-4.39428687e-01,-1.03023911e+00,1.87730461e-01,-3.59216720e-01,6.20508492e-01,-7.73004651e-01,9.66552645e-02,-6.37483418e-01,-9.55420196e-01,3.26788202e-02,-2.66424656e-01,2.82534182e-01,2.52422571e-01],[-1.1397552,-0.50232077,0.09223159,-0.26735067,0.28147846,-0.03696185,-0.20820782,-0.05429426,0.42192706,0.03249143,-0.01698117,-0.23892839,-0.49854392,0.3955738,0.15345843,0.33559328,0.21507952,-0.04064257,0.22699624,-0.50641054,0.07053454,-0.5500075,-0.06001101,0.04517334,0.19567794,-0.17955457,0.27305126,-0.04785448,-0.2612549,-0.35692304,-0.36148024,-0.34148666,-0.4116475,0.27676263,-0.27834377,-0.03701856,0.4965743,-0.55413663,0.3382436,0.08191612,-0.4037471,0.40968558,0.03489543,-1.0914848,0.2991592,0.2770731,0.16211544,-0.25739264,-0.69035816,0.5838545,-0.54167587,-0.356018,-0.07920557,0.48935673,0.07192267,-0.4503314,-0.0949033,-0.8070301,-0.03664298,-0.03012745,0.15907413,0.38043836,-0.6515395,0.05680536,0.44299805,-0.40845385,-0.11105982,0.85363346,0.18670978,0.59739625,-0.13077866,0.4866018,-0.21688354,0.7220425,-0.28363883,0.29003826,0.45031965,-0.2334353,-0.13555112,-0.08814848,0.24942411,-0.1518733,0.21549244,-0.02673819,-0.5664474,-0.35095108,0.27959713,-0.6617079,-0.31726944,-0.04604251,0.25513452,-0.7832588,0.21451592,0.28075683,0.6831345,0.14731511,-0.14408241,0.7662993,0.05382861,0.20411943,1.1278172,-0.3135545,0.34443688,-0.91879576,0.22311266,-0.493602,-0.3179718,0.09849158,0.21863407,-0.21378453,0.05319788,-0.49558386,-0.1182991,-0.8995746,0.6234312,-0.16409193,-0.01116261,0.3130565,-0.06790002,-0.03869436,-0.307492,-0.2618987,-0.4058736,-0.7799788,-0.40722674,-0.05224514,-0.07977924,0.30463424,-0.61879456,0.61510146,-0.44406372,0.1854427,0.07659799,-0.2529513,-0.63536996,0.08354546,-0.7910488,-0.11184791,0.53111285,0.14289203,-0.59229577,-0.84800214,0.49458036,-0.23439384,0.3814895,-0.6320987,-0.646148,-0.51389164,-0.2636328,0.32328245,0.09673677,0.8035153,0.46512341,-0.41475636,0.16931789,0.0594777,0.7246368,-0.573829,0.08537232,-0.6191534,0.468527,-0.05376255,0.06830066,0.00360225,-0.30540186,0.05899662,-0.45022064,0.3874217,0.37411872,-0.17859146,0.57529837,-0.79581916,-0.2667233,0.5157012,0.08399081,-0.15717915,-0.02908161,-0.40732393,0.12727906,-0.0763848,-0.10919992,-0.11197249,-0.30592474,0.13656533,-0.40760893,-0.57018715,-0.45575643,-0.5276779,0.5569771,-0.6172491,0.14456148,0.55341816,0.34557077,0.83609384,-0.5406079,0.2063806,0.34369263,0.24031553,0.44393554,0.14103572,0.2866792,0.51564354,-0.30922356,-0.23026375,-0.01249863,-0.4310589,-0.03728276,0.02892777,-0.3933234,0.3415937,-0.05109423,-0.0038154,0.20525727,0.00766374,-0.2720767,0.15585503,-0.5505235,-0.1660912,0.07791674,-0.05307477,0.32728088,0.10768875,-0.01323222,0.0250519,-0.76231545,-0.36709616,-0.8417117,0.34682631,0.08458699,-0.35436738,0.22781618,-0.30974022,0.26523352,-0.1963718,-0.7568353,0.506031,-0.22253565,0.13102956,-0.14451887,0.7006391],[1.32656544e-01,3.74690861e-01,-6.47610843e-01,5.99962413e-01,9.57974046e-02,-6.69411421e-01,-1.37174418e-02,-3.60853702e-01,-1.28243625e-01,-7.12736070e-01,5.30998647e-01,-2.02095792e-01,-2.66310543e-01,-4.20209229e-01,-5.22028863e-01,1.60498157e-01,1.37574717e-01,-5.29048562e-01,2.95244008e-01,-5.84631562e-01,7.11549520e-01,9.03925672e-02,6.49098158e-02,3.69903386e-01,5.58882117e-01,2.18118384e-01,8.82300317e-01,-3.77407908e-01,-2.91573796e-02,-9.90190431e-02,2.45713443e-01,-3.29108723e-02,-1.14866301e-01,3.43210340e-01,5.58783948e-01,-4.26190466e-01,5.42357266e-01,4.09228921e-01,-3.53431761e-01,-1.51558360e-02,-4.67914253e-01,-1.04333319e-01,-4.00807202e-01,7.89937139e-01,-8.95667017e-01,-5.68698406e-01,-6.62790984e-02,-1.33876249e-01,5.95710501e-02,4.30593431e-01,-2.63106048e-01,-8.59649293e-03,-3.21745962e-01,4.44780260e-01,-3.34984779e-01,4.69491810e-01,-3.77139181e-01,4.55824643e-01,-2.87916034e-01,-2.07307547e-01,1.46476358e-01,-4.36792523e-01,-5.69096357e-02,2.32591882e-01,2.45258704e-01,7.13140249e-01,-5.65890253e-01,-3.69636774e-01,3.38816464e-01,-5.53020298e-01,1.17018059e-01,5.16115069e-01,5.28503597e-01,5.80802619e-01,-5.30845523e-01,-3.12602937e-01,-1.86996922e-01,-1.02932584e+00,-6.94244727e-02,-6.37043834e-01,2.28254683e-02,-3.78890783e-01,2.43339822e-01,-3.41957927e-01,4.81710844e-02,-5.36218047e-01,2.87175745e-01,-2.67661870e-01,-5.33787429e-01,1.42973721e-01,-9.42005068e-02,4.11219224e-02,-3.97981793e-01,-8.38940814e-02,7.41908327e-03,3.63195866e-01,-5.36363244e-01,-7.31270075e-01,-5.15228331e-01,1.02913350e-01,-9.39524099e-02,5.10259978e-02,-4.26278859e-01,-5.73761106e-01,1.51822761e-01,2.78383523e-01,8.92137885e-02,1.90891847e-01,-1.34458035e-01,2.27219269e-01,-2.10177988e-01,4.61378634e-01,-2.23028913e-01,-8.96775723e-02,-5.47655761e-01,-4.69522655e-01,-2.66280949e-01,2.30725408e-01,-6.54265523e-01,-2.06852138e-01,4.91642319e-02,2.10221231e-01,4.90508825e-02,4.00744021e-01,-6.02683246e-01,2.52448112e-01,8.34338367e-01,3.80832970e-01,-1.36269569e-01,-5.65581322e-02,2.15029359e-01,2.05143631e-01,2.13497490e-01,-1.16082504e-01,1.08972490e-01,3.56221974e-01,-4.82870825e-02,2.83694148e-01,3.08143377e-01,5.69819822e-04,3.79897207e-01,-5.92142250e-03,3.94796759e-01,-4.54562038e-01,2.83151358e-01,-3.38788837e-01,2.77896076e-02,-4.12779987e-01,-1.30706936e-01,7.49179125e-01,8.39585438e-03,-2.95011610e-01,9.02766958e-02,3.90737988e-02,1.29719883e-01,3.95794868e-01,3.65572721e-01,-6.22872353e-01,-4.12432134e-01,-9.78780389e-02,-6.21974170e-01,-5.13590813e-01,-6.97078645e-01,-7.67855346e-01,4.33023989e-01,6.55068994e-01,-2.51881033e-01,-2.73098927e-02,2.12242097e-01,-6.07249320e-01,5.51490724e-01,-2.49701574e-01,-3.85519505e-01,-3.10429305e-01,2.02839226e-01,-8.89278650e-01,2.90793121e-01,1.23767741e-01,1.19412489e-01,-5.16098700e-02,6.77378252e-02,-2.57365048e-01,-4.02012736e-01,-5.84925830e-01,-3.71258408e-01,2.37073362e-01,3.40474874e-01,2.76134312e-01,4.48886544e-01,1.98466390e-01,5.56755662e-02,-9.84934047e-02,-8.82389367e-01,1.95762441e-01,4.12135236e-02,-3.68975669e-01,-2.11241320e-01,-4.32436883e-01,-4.17429619e-02,5.84294021e-01,-3.48500088e-02,1.53841883e-01,-5.41672520e-02,-2.77701527e-01,2.23473847e-01,9.26600218e-01,-2.28936628e-01,-4.20234650e-01,-3.93873245e-01,2.37072289e-01,7.79361278e-02,2.37171073e-02,-6.50424421e-01,-2.42540777e-01,2.08967905e-02,4.03809965e-01,6.43345833e-01,-2.94369999e-02,-4.55409847e-02,3.92682552e-02,-6.93618059e-02,-2.14607455e-02,-7.63672814e-02,3.44328098e-02,2.96423048e-01,-2.24595338e-01,-3.52584422e-01,2.34675854e-02,1.09524839e-02,-4.84735191e-01,-8.36884737e-01,-6.58315793e-02,2.28453606e-01,-3.10269862e-01,1.37049134e-03,8.23500007e-02,3.10342908e-01,9.07574952e-01,3.73730600e-01,-9.05053258e-01],[1.91741452e-01,4.44405654e-04,-4.25538599e-01,4.60979849e-01,-9.11141653e-03,3.54116023e-01,-4.35410440e-01,-1.98234394e-01,-9.58705604e-01,-2.84685791e-02,-6.55996725e-02,2.07315966e-01,-5.41544221e-02,3.30341011e-01,-1.64511010e-01,-6.26580238e-01,2.04772353e-01,4.14290369e-01,3.53806257e-01,-5.22772551e-01,-5.41670322e-01,4.39698279e-01,1.51207969e-01,-1.67727813e-01,5.20417690e-01,-8.39780986e-01,2.62916446e-01,8.18761170e-01,5.09900339e-02,-2.14242429e-01,-1.27828836e-01,3.08245957e-01,-3.16209316e-01,5.16225457e-01,-5.06845899e-02,-3.15030217e-01,-3.46774548e-01,4.54920143e-01,3.49568963e-01,-2.51223922e-01,7.10052326e-02,-2.12696522e-01,-7.60714471e-01,4.70193714e-01,2.45769650e-01,-2.57415622e-01,2.08894424e-02,-5.44040143e-01,1.57386333e-01,6.47852600e-01,5.17346971e-02,-4.05680776e-01,3.94587249e-01,3.76607120e-01,8.96452516e-02,-1.70496866e-01,-1.51504725e-01,5.36379516e-01,-1.86963633e-01,-2.72057921e-01,3.95275116e-01,2.03009114e-01,9.91513506e-02,3.67347807e-01,-4.75859374e-01,-9.41075981e-01,-3.39962006e-01,-1.22397459e+00,-6.78302720e-02,3.18249650e-02,-6.86506331e-01,7.77404666e-01,-2.60544531e-02,3.52944642e-01,6.01653643e-02,3.21570486e-01,2.02841207e-01,-9.95056033e-02,-5.05674630e-02,5.31506658e-01,-4.34038699e-01,1.67729974e-01,-2.23942518e-01,-9.54656601e-02,-5.26630171e-02,-4.70563322e-01,-3.74620169e-01,2.64579803e-01,1.59985185e-01,-9.57847834e-02,-1.65583938e-01,-7.37171695e-02,6.78428337e-02,-2.96679556e-01,3.90606850e-01,-9.98268872e-02,5.16445935e-01,-2.28062823e-01,-4.14078027e-01,-1.97222546e-01,8.46939757e-02,-5.11601754e-02,8.44385028e-02,-9.37504470e-02,-5.29606901e-02,-3.31755131e-01,-1.43355817e-01,-2.16091856e-01,-5.01994550e-01,5.58349788e-01,-1.42108306e-01,1.06401786e-01,-6.13722146e-01,-3.50220382e-01,-2.22356170e-02,-3.28627765e-01,7.73725808e-01,-2.51269937e-02,7.48929262e-01,-6.47816509e-02,7.93580174e-01,5.38585901e-01,-3.22568476e-01,-2.53204512e-03,4.51165706e-01,2.99201578e-01,2.29275972e-03,-2.26959780e-01,-2.05363721e-01,-4.17706490e-01,9.52161670e-01,-3.87108773e-01,1.31302699e-01,-2.06995413e-01,-6.84772968e-01,-2.43744105e-01,-5.26822984e-01,-4.06395793e-01,-6.12747788e-01,-1.19735992e+00,-6.57647133e-01,4.44339782e-01,-3.92095178e-01,4.73810464e-01,5.90988874e-01,-9.54471469e-01,-1.42858908e-01,1.80578709e-01,1.09403105e-02,-7.46732414e-01,1.95436731e-01,4.12653208e-01,2.09468544e-01,1.21209353e-01,4.85232711e-01,2.18562871e-01,6.95907414e-01,4.03638422e-01,1.82264149e-01,-4.08602804e-01,-1.87771678e-01,1.95782125e-01,-1.16842553e-01,2.44323760e-01,6.41291738e-02,2.30252564e-01,4.21835899e-01,2.58089125e-01,4.82042015e-01,4.03990060e-01,2.46456578e-01,3.57505947e-01,3.25313956e-01,-2.37711102e-01,8.45981836e-02,-3.49451274e-01,-7.57859826e-01,6.77879810e-01,2.25420401e-01,3.27215970e-01,-7.45911896e-01,-2.65777498e-01,-1.39811441e-01,-4.84172642e-01,-1.54042974e-01,2.25795627e-01,7.37835094e-02,5.19831419e-01,-1.47744909e-01,-6.89184889e-02,8.33929360e-01,1.97926238e-01,-4.59612101e-01,3.67333859e-01,-3.96773100e-01,4.99303371e-01,1.08495438e-02,-1.28639653e-01,3.27037364e-01,-9.73224461e-01,-1.90873027e-01,-4.91582118e-02,-3.89303684e-01,-4.38541442e-01,5.09027421e-01,-2.76389211e-01,4.35182482e-01,-3.04791123e-01,3.04775178e-01,-2.59180218e-01,-4.11941022e-01,-2.43171543e-01,-6.44885227e-02,-3.45777392e-01,4.83814865e-01,-7.76612818e-01,1.42538726e-01,3.67438346e-01,-6.99674934e-02,3.29373270e-01,-4.28114831e-01,-4.01267409e-01,-1.41381755e-01,-1.82890519e-01,-3.03503513e-01,1.46224260e-01,4.02081341e-01,5.03896534e-01,-6.98562801e-01,7.85163119e-02,-7.24544972e-02,7.55335614e-02,2.62171209e-01,-6.48507357e-01,-2.45665997e-01,4.37688291e-01,2.05014452e-01,-1.49043649e-02,1.31322578e-01,-7.40045846e-01],[0.5480669,0.0657418,0.38078588,0.68701243,-0.5385719,-0.22859755,-0.40806404,-0.19076249,0.52079993,-0.07341328,0.48084846,-0.15699981,0.17848259,-0.11362629,-0.6948029,-0.26375994,-0.35064024,0.03404541,0.11094151,-0.2437265,0.307633,-0.5133083,0.37947318,-0.0491284,-0.45336655,-0.11076704,0.15508041,0.81791365,-0.1540035,0.253477,-0.3251734,-0.16472714,0.25824496,0.0970108,1.264761,-0.14136773,-0.18109508,0.15703896,-0.7257976,-0.2777064,-0.7369485,0.4330739,-0.00861591,-0.07888339,0.86241686,0.71848357,0.7489886,0.81274945,-0.33425373,-0.13528469,0.2677271,0.2528606,0.21174368,0.03658979,0.82518476,0.05965647,0.88553935,-0.04341239,0.32058772,0.2270998,0.13068137,0.49707395,0.34879062,0.43658882,-0.37975028,-0.2509817,-0.8439365,-0.18630621,0.09424753,-0.4228214,0.04321646,-0.08970854,-0.05533147,0.08132693,-0.35048613,0.14905652,0.15577783,0.06663935,0.27246892,0.05188813,-1.7717949,-0.21569942,0.14703701,-1.0534238,0.25260702,0.22766078,0.1777259,0.16764867,-0.102352,-0.20705879,-0.27016684,-0.12166568,-0.44438067,-0.01858898,-0.42204309,-0.03349221,-1.0547631,-0.3964591,0.01707424,0.15625048,-0.8571781,-0.12615961,-0.48232886,-0.14468801,-0.13655443,0.09140982,-0.1002572,-0.5108025,-0.39092806,-0.36068872,-0.28407416,0.21182832,-0.7108842,0.31758893,-0.3403787,-0.64298606,-0.04472863,0.57258266,0.12166771,0.69911265,0.00866169,0.4753874,0.253826,-0.2562373,-0.6375896,-0.31062752,-0.6477973,0.03634868,-0.43591338,-0.10695519,-0.85355175,0.15073034,-0.02090044,-0.2298264,-0.11830106,-0.17306451,-0.19008408,-0.44036564,-0.3150069,0.66969115,-0.11959147,-0.25921828,0.07226536,-0.3826756,0.26580212,-0.9165643,-0.3421176,0.22445242,0.49987957,-0.23020026,-1.0496927,-1.005855,0.01562674,0.58752203,-0.14438714,-0.1561992,-0.02045785,-0.4541857,0.22525808,0.07186435,-0.06279762,-0.29359278,-0.08360578,0.59174687,0.37724918,-0.6877257,0.05103741,0.40592974,0.9573966,0.06840289,0.12089962,0.3261812,-0.27704358,-0.1887488,0.36377463,-0.18554026,-0.3885203,-0.0184504,-0.23506321,-0.33696055,-0.46870694,0.34747598,-0.017399,-0.3422878,0.04135449,0.29252282,0.55455536,0.57418853,-0.2839681,-0.37572488,0.23611163,0.01102649,0.49275732,0.20929308,-0.12444305,0.31343666,-0.04888831,-0.2136806,0.38324603,-0.6182701,0.04769949,0.9829735,-0.32928184,-0.17881435,0.2427987,-0.25851586,-0.19089426,-0.146787,-0.08664127,0.29696828,-0.02049725,0.11285683,0.5358991,0.3211265,0.5319625,-0.5652282,-1.0344332,0.3925341,0.15810134,0.30055484,-0.36381352,0.25868046,0.05960619,0.11487611,-0.04791497,0.81993264,-0.586436,-0.14223985,0.03599095,0.04050918,-0.01583096,-0.3949102,0.2143622,-0.70713794,0.48409027,-0.22899953,-0.7599893,-0.31635728,0.75151235,-0.45823985],[-1.18862972e-01,4.52469945e-01,3.79783541e-01,-7.42815211e-02,-1.57589972e-01,-8.44056718e-03,3.91454637e-01,6.21487856e-01,1.82503745e-01,-4.13488746e-01,-6.11236580e-02,5.97868800e-01,2.09931254e-01,-1.98569611e-01,-6.52476072e-01,-2.51505703e-01,2.69729912e-01,4.80850965e-01,-1.41162828e-01,-6.17177725e-01,1.14137784e-01,-4.99212325e-01,2.38497823e-01,2.52027869e-01,-1.71691030e-01,-1.86362252e-01,2.50538345e-04,3.35866064e-01,-1.89124029e-02,5.48124790e-01,9.59488094e-01,-4.11304794e-02,-9.57747459e-01,3.56825799e-01,1.81559175e-01,2.18105853e-01,-9.55819618e-03,-5.89963317e-01,-3.33839804e-01,2.05698252e-01,-1.35402121e-02,5.59214413e-01,-9.75174364e-03,1.85875475e-01,-4.81347352e-01,-4.35701758e-02,-9.84303474e-01,-2.66641736e-01,-2.20027775e-01,1.93767995e-01,4.32466984e-01,-3.50190759e-01,-7.04440296e-01,7.85089433e-02,-3.27690214e-01,4.16548461e-01,2.10136071e-01,1.62350222e-01,-1.98837128e-02,7.29451716e-01,1.24539264e-01,-5.37822306e-01,2.57829815e-01,-4.47729319e-01,5.43607295e-01,2.74447381e-01,1.62051857e-01,4.29527491e-01,6.31067604e-02,-2.61280239e-01,4.32331145e-01,4.91965413e-01,3.43262941e-01,-3.51167351e-01,3.53377521e-01,4.12511021e-01,-3.51300031e-01,1.74376041e-01,2.41382018e-01,-5.87368667e-01,-2.05104649e-02,-1.12393044e-01,1.94276974e-01,-1.90407723e-01,2.31387198e-01,3.06933582e-01,1.02371098e-02,-2.50275075e-01,1.02739356e-01,3.04599196e-01,-1.99703321e-01,-5.29906213e-01,-2.03488037e-01,-1.72493473e-01,-7.05294460e-02,-1.78025842e-01,-3.85594398e-01,-7.08733320e-01,6.73697114e-01,8.32054839e-02,3.14297318e-01,-6.19950712e-01,-2.57440388e-01,-4.96906936e-01,3.66653502e-01,5.21361351e-01,-1.46238253e-01,-9.56736729e-02,1.96015951e-03,1.95337743e-01,-7.47550309e-01,1.22044764e-01,-1.75129354e-01,-2.50589997e-01,-7.81633914e-01,-5.10085225e-01,2.22231876e-02,2.57617608e-02,1.28873289e-01,-1.18491612e-01,1.67198598e-01,9.58595634e-01,-3.45359266e-01,-2.58179396e-01,-2.38958284e-01,-1.74328595e-01,-7.20170617e-01,-2.81327188e-01,2.66980249e-02,-5.95346093e-02,2.87808981e-02,2.14773580e-01,-6.60578489e-01,2.73330510e-01,1.15091197e-01,-5.60487248e-02,5.08342803e-01,1.45747125e-01,4.03293461e-01,-3.84925723e-01,-4.42236334e-01,-2.47209445e-01,-3.10685307e-01,5.24511814e-01,-1.25494570e-01,-1.11257419e-01,-3.75461578e-01,4.34063137e-01,9.85145643e-02,-8.24825346e-01,2.16612309e-01,-1.87227786e-01,1.89368427e-01,7.37494588e-01,-7.89884552e-02,4.72770900e-01,4.26738650e-01,-2.24476531e-01,8.90657067e-01,-7.74372518e-02,-5.69522344e-02,-9.53824759e-01,3.03225219e-01,-9.85306382e-01,7.40862429e-01,-1.18853197e-01,1.22981571e-01,2.04990387e-01,2.49465331e-01,-3.21168959e-01,6.15985282e-02,-3.55922170e-02,1.75945044e-01,2.38647386e-01,7.97833353e-02,-2.86017925e-01,1.88898936e-01,-1.21246509e-01,1.33985728e-01,-3.44980508e-01,1.09639680e+00,-4.44944808e-03,2.92288780e-01,-1.65316045e-01,-4.24981378e-02,-1.22441001e-01,-6.52962446e-01,2.21764043e-01,-2.26756528e-01,-6.21611834e-01,-1.47162870e-01,4.24657762e-01,-3.62973154e-01,3.97586495e-01,3.94749582e-01,-3.88064310e-02,4.19983506e-01,-2.23288313e-02,3.43331039e-01,4.05355066e-01,-8.57917219e-02,6.96356237e-01,-7.42545962e-01,2.76131444e-02,-2.67942220e-01,4.16680388e-02,4.37294036e-01,-8.06945711e-02,1.87167645e-01,2.24179998e-01,2.55156696e-01,1.35971069e-01,1.83206260e-01,-4.05730933e-01,4.46372092e-01,-1.07843250e-01,-1.39348269e-01,-1.73737571e-01,-9.99617994e-01,4.99503136e-01,5.62178791e-01,-4.65502977e-01,-6.67404532e-01,-2.58423737e-03,-3.94725651e-01,-1.28718901e+00,3.80849838e-01,-4.94914025e-01,4.29536514e-02,9.25348327e-02,2.11142391e-01,8.99489641e-01,4.74753641e-02,2.03648895e-01,6.37522340e-02,-3.37921828e-01,-2.16012716e-01,6.98789001e-01,2.39104629e-01,4.05153772e-03],[-0.15365134,-0.5140609,-0.49043277,0.42666733,0.6281119,0.23584124,0.5276892,-0.20724003,0.04612149,0.02971226,0.09723334,-1.065323,0.32059735,-0.11533749,-0.47797605,-0.40423113,-0.923158,-0.1355837,0.3434505,0.19774118,0.88251394,-0.21859819,0.27050558,-0.27329692,0.44919983,0.36070764,-0.02350434,-0.5459956,0.05377505,0.0158663,-0.274074,0.33771554,-0.39254063,-0.50928676,-0.59456116,-0.31611267,-0.01983227,-0.01789855,0.06121812,-0.3731818,0.20991766,0.39821425,-0.04554138,-0.7840181,0.01473422,-0.00671488,-0.53264,-0.27867863,-0.25045776,-0.39855528,-0.39443862,-0.31514063,-1.082662,-0.06855719,0.20040905,-0.6481238,-0.29279265,0.23626946,0.39595786,-0.1178173,0.41325435,0.17377704,-0.32565996,-0.36674365,0.11336269,0.13380425,-0.12929295,-0.32967564,-0.3523658,-0.09294567,0.62846893,-0.30011472,-0.19335999,-0.07555539,0.15021075,0.708627,-0.18745011,-0.57075894,-0.26181722,0.3171211,0.17792165,-0.20944375,0.2001934,-0.47056815,0.38339272,0.41776386,0.07167882,0.35168406,-0.21707135,0.25564358,0.24092838,-0.5623609,0.04076586,-0.27884376,0.03855708,-0.6789994,-0.08446243,0.1719568,-0.301632,0.28696284,-0.6041347,-0.32375312,0.18959896,-0.08755246,-0.29386652,-0.2424829,-1.0125594,0.79364485,0.19931707,0.09869538,-0.12429106,0.13370696,0.02530433,0.26018485,-0.1974381,-0.18511353,-0.2277268,0.5541442,0.12323558,-0.43600148,0.11970072,-0.27160463,0.7120562,-0.86202884,0.37205154,-0.01620439,-0.04337901,-0.59400165,-0.5498463,-0.21182439,0.34533343,-0.7949454,-0.3983564,0.22349276,-0.65563023,0.00279601,-0.78678036,-0.5302431,0.16929054,-0.6375504,-0.32719544,-0.177119,0.17701454,0.3830257,0.81810206,-0.54826003,-0.2081844,-0.01872217,-0.25915498,-0.18039624,0.30481055,0.18846044,-0.3646181,-0.45792064,0.4506131,-0.3693342,-0.1916108,-0.1785283,0.17143106,0.38949353,-0.04179508,0.04535574,-0.38028312,0.4258565,-0.46369964,0.8422098,-0.02471538,0.38822725,-0.04721221,0.21121545,0.449018,-0.16916895,-0.2042827,-0.06980913,-0.1810949,-0.11778704,-0.32580435,0.63880366,-0.00355144,-0.29222602,0.10173851,-1.2783084,-0.54271376,0.29248634,0.24222898,0.11784635,-0.5868308,-0.46971858,-1.3598888,0.6251001,-0.3570934,-0.25900802,0.55716807,0.1985544,0.20928112,0.02402399,0.19860592,-0.07094819,-0.3062882,-0.45872188,0.22095627,0.3368756,-0.5586108,-0.18577795,-0.49303076,0.15961535,-0.21364573,-0.20948993,0.1922292,-0.09067923,0.132391,1.2597897,-0.07321329,0.24068736,-0.16848046,0.57001257,0.3196154,-0.6689491,-0.9122036,0.36142784,0.45095193,0.16918536,0.23855026,-0.3596756,-0.11295211,0.71150166,0.4575563,0.2570333,-0.2125654,-0.34959525,0.35342017,-0.80596256,-0.4051231,0.06641247,-0.15273051,-0.05177113,-0.6299135,0.05697516,-0.19045149,0.05054918],[-0.2350923,-0.04097695,-0.59289503,0.14978273,-0.5440514,-0.30748665,-0.08832303,0.14016873,-0.12889132,0.1633464,0.12400462,-0.56852806,-0.42478457,0.08686842,-0.8243114,0.17944005,-0.01644148,-1.0834819,-0.02002969,-0.49037892,-0.23658612,-0.8006224,-0.02999929,-0.5424841,-0.03564496,0.06424834,-0.3303443,0.32451907,0.5049147,0.35594317,0.3165214,0.17208143,-0.39528343,-0.27581957,0.01896267,0.36868602,-0.26105493,-0.30294707,-0.08645318,-0.14133045,0.12033123,-0.13705724,-0.29712406,0.21615686,0.19448054,0.14708583,-0.36507976,-0.2281317,-0.5404159,0.59643257,-0.32640356,-0.03189527,-0.4921073,-0.20331644,-0.52568257,-0.02729818,-0.03307479,-0.1962617,0.71972275,0.2020097,-0.3031336,-0.17384882,-0.5148201,0.366164,-0.10537895,-0.5609912,0.4001478,-0.33832428,0.93437314,-0.73060423,0.9080687,-0.75510687,-0.08646979,-0.4303572,0.22862883,1.1645963,0.26920256,-0.32913426,0.54245293,-0.16468634,0.37794515,0.24125299,-0.24053021,0.0036583,-0.18229793,-0.38571364,0.26689398,-0.009456,0.5177289,0.10414746,0.24865222,0.22580792,-0.26909456,-0.42188042,0.15179294,-1.2133133,-0.58220893,-0.23477368,0.05017471,0.2522329,-0.39888483,-0.87087125,-0.6194988,0.5011239,-0.29140207,-0.59730196,-0.515253,0.13138922,0.14395872,0.20666423,0.37663665,-0.20168604,0.47944373,-0.04574366,-0.25853702,-0.72418904,0.2422077,0.72093374,0.10113752,0.47928575,0.37719324,0.5775461,-0.04200065,0.67784816,-0.7259988,0.17410482,0.21987565,-0.18382211,0.07784486,-0.25371653,0.05647541,0.3526584,0.14285606,0.08289111,-0.23513952,0.842755,0.5894651,-0.21687257,-0.5898846,-0.1918142,-0.34613746,0.4650853,0.11805297,-0.6219197,-0.7750753,0.28103533,0.17884356,-0.3084127,-0.2885042,-0.36339685,-1.1546376,-0.91257846,-0.71531314,-0.66347367,-0.18735754,-0.8573782,-0.35429388,-0.736295,-0.71539867,0.5453903,-0.10456858,0.33505893,-0.67892015,0.57037264,-1.0098761,-0.5400712,-0.260906,0.33190957,0.4008116,-0.40111464,0.3304452,-0.18208258,-0.25731272,-0.25121462,0.17217529,-0.40506214,0.6296112,0.03150245,0.05427152,0.2956674,0.09310342,0.04662264,-0.12765163,-0.12103361,0.28367162,0.00429857,0.9829175,0.33349028,-0.2559511,0.862831,-0.10285928,0.02728708,-0.14833418,-0.40677866,0.52088785,-0.14576925,-0.49263653,0.3218314,-0.241225,0.05245866,0.00498665,-0.6227596,0.43332195,0.2882676,-0.55847687,-0.5757048,-0.26577997,0.63105744,0.9276749,0.05146658,0.31062263,-0.13811643,0.3260784,0.27035764,-0.78934646,-0.30716076,0.65857095,-0.0588175,-0.49640247,0.09212018,-0.2580815,-0.12446238,1.019179,-0.06655574,0.30401552,0.676577,-0.07177006,0.30491284,-0.5804623,0.22485545,0.75950843,-0.31640303,0.03386886,0.56794655,-0.32158673,0.24280654,-0.16627549,0.1582166,-0.11572744,-0.13057281],[-0.06284013,0.16747248,-0.06955961,0.02207351,-0.14804383,0.4021633,0.20545675,-0.44730186,-0.10581057,-1.2421286,-0.2956871,0.21116984,0.10411704,0.1857165,-0.45725623,-0.03386748,0.18353522,-0.7803468,-0.13085446,-0.39924553,-0.202715,-0.0053978,0.00265313,0.21845151,0.4500026,-0.7538076,0.36047375,-0.05976154,0.20150001,0.45605898,0.64452195,-0.00450852,-0.31092647,0.39912063,-0.26226693,-0.22725208,0.34529588,0.06707665,0.687177,-0.35270366,0.8826847,-0.81699115,0.30969864,-0.81584185,-0.01807751,-0.00528334,0.15607876,0.70750463,0.29481912,-0.29592225,0.13311988,-0.00388992,0.47105345,-0.4091425,-0.1902496,0.2456863,-0.9548792,-0.8602253,0.44180286,-0.15918207,0.05320411,-0.49112958,-0.07431621,0.08634268,0.58527845,-0.8789918,0.8107478,-0.06517871,0.09586215,-0.12284417,0.39500934,0.24588524,-0.38088578,-0.44212887,0.62619895,-0.14684676,-0.41172174,0.21392412,-0.61740595,0.21041584,-0.4551209,-0.03336025,-0.36455056,-0.21839994,0.24047202,-0.4165801,-0.0354447,0.6684382,-1.1486415,0.7721756,-0.04735201,-0.45815632,-0.49851516,-0.00223309,0.12256426,-1.0316786,-0.7801637,-0.0919238,0.426667,0.12618813,-0.15625015,0.466924,1.0400685,-0.3200616,-0.05643502,-0.017447,-0.4469637,0.24252586,-0.55004966,0.29197523,-0.4651836,0.27884,-0.08701807,-0.01441852,-0.16305915,-0.45702484,0.01752383,0.21569064,-0.3751164,-0.6240981,0.2476522,-0.75803024,0.35439882,-0.28721765,-0.20169614,-0.0306648,0.0190716,-0.06946537,-0.30189958,0.41929504,-0.67086524,0.51048046,-0.95592,-0.16964234,-0.00925957,0.21038142,-0.60409415,-0.07050423,0.2777089,0.29594085,-0.24976695,0.23424824,-0.02418415,-0.3322433,-0.403662,-0.14919811,0.00752499,-0.04058741,-0.29510912,0.05828332,0.31199345,0.13000156,-0.13236658,-0.5031315,0.19657658,-0.64102197,0.5278681,-0.32212895,-0.10561553,-0.8471047,-0.41673434,0.22895528,-0.26220238,0.1010342,0.33826932,-0.38201287,-0.00305777,0.10693232,0.35844794,-0.09137894,0.2726809,0.2879577,-0.6340502,0.22826901,0.15166055,-0.09511635,-0.7121487,0.19868276,0.11623585,-0.10546729,0.14894857,0.29595444,-0.64620614,0.05457152,0.26111865,0.3241202,0.14290085,-0.1279845,-0.22503237,-0.863691,-0.32283977,0.3805013,-0.30077118,0.5029166,0.26901436,0.19161415,0.04074844,0.12096763,-0.7772472,0.4261597,-0.6845093,-0.22985293,-0.35762858,1.0636936,-0.1639205,-0.6355244,0.23246008,0.22704497,-0.17895424,0.59839636,-0.12002154,-0.0139578,0.678314,0.15473372,-0.03571888,0.19580528,-0.5079559,0.8685,0.06131,-0.25303683,0.494909,-0.34983656,-0.30211267,0.14877552,0.5175115,0.75581795,0.2581477,-0.0360227,-0.41148126,0.3945492,0.22169147,-0.5244435,0.02215431,-0.07235546,0.44933546,0.15672156,-0.4354101,-0.2579032,0.15638462,0.286338],[7.05229193e-02,-5.60871899e-01,2.84248114e-01,-1.39768362e-01,-9.15248245e-02,2.66091496e-01,4.42717105e-01,2.57567495e-01,-8.16361606e-01,-1.55061319e-01,-2.03121617e-01,-2.13709965e-01,-2.14748934e-01,-1.24645151e-01,4.99986708e-01,-1.76294625e-01,-1.79853365e-01,-3.17158997e-01,1.89768299e-01,1.24730028e-01,-8.72491241e-01,-4.41045612e-01,3.46135139e-01,-1.85449302e-01,-1.42742705e-04,2.09936760e-02,-2.99651295e-01,6.69969022e-02,-3.04395497e-01,1.01329744e+00,-5.25403023e-01,4.61441159e-01,-4.38510150e-01,5.12324035e-01,-4.77370769e-01,8.11337948e-01,3.37450653e-01,-4.39123511e-01,-5.88800199e-03,5.58516145e-01,2.09928825e-01,-1.07113747e-02,-3.40623379e-01,3.29428971e-01,-9.17390734e-02,-1.95935681e-01,3.55709851e-01,2.43576065e-01,-1.08382635e-01,5.52728176e-01,5.10154188e-01,-7.62800336e-01,-1.80164188e-01,-2.70912871e-02,-3.28282684e-01,-4.84298915e-01,-2.28674173e-01,-2.93662250e-01,4.64013591e-02,1.46065637e-01,6.42121434e-02,-3.38476628e-01,-9.79350060e-02,-1.80164874e-01,4.87069339e-01,2.30279624e-01,-7.44780060e-03,2.82813251e-01,2.63566971e-01,-5.64407527e-01,4.30744588e-01,2.07345784e-01,3.95412669e-02,-7.60213077e-01,1.60949796e-01,2.85314649e-01,-2.07691506e-01,7.96125308e-02,3.08677197e-01,1.75695438e-02,-1.05915241e-01,7.65441777e-03,9.79727209e-02,2.83126742e-01,2.01939791e-01,2.81137556e-01,2.63226211e-01,-7.21220151e-02,4.64769602e-02,1.35829076e-01,2.15735156e-02,-1.66251689e-01,-4.69093531e-01,1.25857636e-01,2.98196040e-02,-4.51453269e-01,4.52087849e-01,2.57731825e-01,-3.60509902e-01,-4.06289458e-01,8.93097594e-02,-2.25975260e-01,8.80929306e-02,-5.80322683e-01,8.85254979e-01,-7.08174482e-02,1.25090882e-01,-1.63618132e-01,1.18108213e-01,9.43595767e-02,3.64716440e-01,2.31752470e-01,3.13155442e-01,2.97356009e-01,4.79758391e-03,1.61035433e-01,2.52985924e-01,7.41175354e-01,-4.51826081e-02,1.33808032e-01,1.74248144e-01,7.72419751e-01,4.33140755e-01,-7.72897124e-01,-4.19403911e-01,-8.26871321e-02,3.51545289e-02,-7.56485343e-01,3.60067755e-01,1.04201607e-01,-4.08368737e-01,1.42628746e-02,-4.26979326e-02,-1.03963017e-01,-9.16909650e-02,-4.97104019e-01,2.50591159e-01,4.11506563e-01,2.60690507e-02,1.19892247e-01,6.60308171e-03,2.28688613e-01,2.15647236e-01,-5.64765275e-01,3.15373063e-01,1.22075438e-01,4.01422888e-01,-4.10474658e-01,-5.51007450e-01,2.39128098e-01,2.67673463e-01,3.80887657e-01,-5.66297352e-01,-3.01009268e-01,-5.60945451e-01,2.56880373e-01,-1.85816791e-02,3.80078942e-01,6.91693604e-01,-1.67011827e-01,2.48560123e-02,4.11822915e-01,1.68589234e-01,-3.77602100e-01,3.50601852e-01,4.05528158e-01,-4.17329893e-02,4.18328196e-01,-1.19394504e-01,4.05121952e-01,4.50599760e-01,2.49591276e-01,-5.13381183e-01,-5.76406643e-02,-1.49923265e-01,1.08808257e-01,-1.20768540e-01,-2.52976269e-01,-9.35929939e-02,-6.33899868e-01,8.91153753e-01,1.63108036e-01,2.50463217e-01,-3.40172380e-01,-1.64006442e-01,-5.33688962e-02,6.87045097e-01,6.21121526e-01,1.01062283e-01,-3.91819179e-01,-2.49302328e-01,1.94218993e-01,1.19071580e-01,5.86995273e-04,-4.03332651e-01,8.18944037e-01,-3.75822842e-01,1.43118426e-01,-1.34952855e+00,-1.46946862e-01,9.12021041e-01,-1.06947649e+00,4.49941933e-01,3.33988994e-01,-3.51536721e-01,-4.65541817e-02,1.40491933e-01,-3.25198889e-01,-3.94720852e-01,7.27725625e-01,4.47992533e-01,-5.57920277e-01,1.36482239e-01,4.27976727e-01,-4.84803289e-01,4.62837756e-01,-4.50647950e-01,3.26361269e-01,9.81072038e-02,-1.08301960e-01,-3.88219655e-01,-7.96770036e-01,8.13763216e-03,3.23086947e-01,1.46892980e-01,-8.11774433e-02,2.15997353e-01,1.30951673e-01,-4.07204181e-02,-5.65134764e-01,-3.42010975e-01,3.40860099e-01,-1.62553340e-01,6.48300648e-01,-1.11631006e-01,-1.22368908e+00,6.06133379e-02,-2.40138788e-02,-8.34687233e-01,3.26347172e-01],[-8.28659773e-01,3.88312966e-01,-1.07969522e+00,7.98220783e-02,7.68520236e-01,1.84697449e-01,-1.24662407e-01,-1.71272591e-01,-4.47432280e-01,1.41020328e-01,-5.59078716e-02,-7.50172973e-01,-2.51098692e-01,-2.68870126e-03,-3.66554201e-01,6.04669869e-01,-3.37359548e-01,6.10294819e-01,6.05451822e-01,2.00094402e-01,-4.04691726e-01,-1.45185351e-01,-3.67662519e-01,1.28187329e-01,-6.89901412e-01,1.67823918e-02,2.60228693e-01,-4.51478958e-01,5.97098887e-01,-8.10182750e-01,-1.48482531e-01,-7.82349706e-02,4.26135570e-01,2.26752862e-01,-5.69549143e-01,2.51911759e-01,1.06975332e-01,-1.37125522e-01,-3.31777126e-01,-5.78591704e-01,-1.42357957e+00,1.17306314e-01,-2.01565012e-01,-2.91712016e-01,2.26770476e-01,2.57297307e-01,-2.70881653e-01,4.18780923e-01,1.30498037e-01,2.26551428e-01,2.86066979e-01,-4.80810881e-01,-2.29355991e-01,-5.69444418e-01,-2.54011042e-02,-1.06872940e+00,1.91706330e-01,-2.07086414e-01,-1.38921130e+00,7.25682855e-01,-1.76114857e-01,-6.81034923e-02,-1.76557332e-01,-1.68152913e-01,6.98028684e-01,5.49992144e-01,6.30973518e-01,-2.62143016e-01,1.03425927e-01,-1.35392100e-01,1.24370098e-01,1.29509911e-01,-4.48872238e-01,3.12280327e-01,-6.44680038e-02,8.71416628e-01,-6.10399544e-01,5.75926602e-01,-8.08806837e-01,-4.53305513e-01,7.34197915e-01,-5.76192178e-02,-1.05317347e-01,-3.37913632e-01,6.41936004e-01,6.90990627e-01,-9.08540189e-03,-4.25400078e-01,-1.79071426e-01,-6.46612287e-01,3.83753955e-01,-3.06656986e-01,-4.58792895e-01,3.40500474e-01,-5.52717328e-01,-5.24329484e-01,2.80968964e-01,2.58122861e-01,3.56702089e-01,-6.92763478e-02,1.32904127e-01,1.98570342e-04,-3.55110228e-01,3.67525995e-01,-3.09589893e-01,-2.37737164e-01,5.14332712e-01,-6.32617414e-01,-3.25671434e-01,-7.59548992e-02,-3.45216870e-01,8.06993097e-02,-9.58410501e-01,-4.92060512e-01,4.67880785e-01,-2.40374744e-01,-6.40765548e-01,-1.09432375e+00,-2.40893736e-01,-5.60043931e-01,-4.08238947e-01,2.63650596e-01,1.38543710e-01,4.45949137e-02,2.65463740e-01,-5.22829354e-01,4.86836135e-01,1.50991920e-02,9.51118544e-02,-7.51589298e-01,-9.77689177e-02,9.04522657e-01,-2.32503891e-01,3.19804877e-01,-1.52961090e-01,2.16775179e-01,6.66992962e-01,-1.60867110e-01,6.96594894e-01,-5.16079552e-02,1.54282793e-01,3.73964965e-01,-9.10919666e-01,2.87299216e-01,-3.22414428e-01,-2.04475403e-01,2.68163174e-01,-2.36255080e-01,2.25587904e-01,-4.42289382e-01,3.18309098e-01,5.79004958e-02,-6.38991892e-01,-6.94639757e-02,-3.15919071e-02,3.78204972e-01,-7.54390538e-01,3.35847944e-01,4.03984226e-02,-6.52406514e-02,-3.37364018e-01,-3.31257358e-02,9.04355526e-01,-5.12355715e-02,1.11134688e-03,-3.61356944e-01,5.06098390e-01,5.33785701e-01,4.28074419e-01,-2.15500385e-01,-3.38822424e-01,-2.95609832e-01,-6.32598639e-01,-1.52375977e-02,-7.23181307e-01,-1.29058778e+00,-4.86407101e-01,-1.20911166e-01,1.14054173e-01,9.08417702e-02,5.26975058e-02,-2.57035881e-01,-1.81470603e-01,-3.61019641e-01,-6.79989755e-02,6.58727348e-01,-6.59841716e-01,-1.25568599e-01,-6.08504295e-01,1.84959546e-01,4.97827977e-01,3.91674414e-02,-3.90107542e-01,2.19714507e-01,-4.33369547e-01,-1.50603633e-02,1.12333000e-01,3.75268936e-01,-1.07505418e-01,6.18376210e-02,-4.84539628e-01,-4.93719667e-01,6.62011623e-01,1.54861763e-01,-2.41315290e-02,1.33713812e-01,1.38231769e-01,-1.26596284e+00,6.10060751e-01,-1.90862983e-01,-4.78395373e-01,8.51204395e-01,-8.38603914e-01,-9.76949096e-01,-8.41916323e-01,-2.23437175e-01,2.18320623e-01,2.82593727e-01,4.58565980e-01,2.12194055e-01,-1.36393416e+00,-1.46121979e-01,2.13241220e-01,-2.90392041e-01,2.23614857e-01,4.61736590e-01,6.70877337e-01,-3.33351195e-01,2.65360743e-01,-1.62627906e-01,-5.93431413e-01,-7.96848357e-01,1.01500547e+00,1.26327351e-01,2.13008463e-01,4.35774177e-01,7.16136813e-01,4.32571024e-02,1.19090058e-01,3.75430435e-01],[-0.16652955,0.44350025,0.23879103,-0.29195255,-0.60318774,0.37214085,0.18700862,-0.10202877,0.365543,-0.10259203,0.36404136,-0.58643746,0.17600101,-0.2285426,0.48094508,-0.30699968,-0.1788638,0.07041278,-0.20528468,0.02365574,-0.02243446,0.38488913,-0.04468666,-0.5035083,0.60816145,0.09366433,-0.5752363,-0.0452144,0.06520574,-0.57750326,-0.23237815,0.41846088,0.32686564,0.30525336,0.24790932,-0.01038643,-0.08996053,0.132827,-0.12224194,-0.48530272,-0.10672593,0.20164873,0.10984464,0.21412733,0.00345774,0.08136646,0.29645392,0.42702222,1.2505671,0.02947392,-0.46882403,0.4835111,-0.01807907,-0.45072585,-0.00735797,-0.7573098,0.24191071,0.02316563,0.63052636,0.27767244,0.27064872,-0.3020398,0.36106622,-0.7661472,0.21465708,-0.18524091,-0.8334489,-0.09006038,-0.5130725,-0.2219217,1.1735376,-0.07488465,0.31173453,-0.47482738,-0.32503572,0.38355798,0.655982,-0.02652642,-0.49643254,0.5350808,-0.24465089,0.21835242,-0.09837603,0.47095612,-0.8589351,-0.03078356,0.13280556,-0.7806419,0.17662255,-0.20078543,0.54338396,-0.19115455,0.5840022,-0.2813855,-0.08451702,0.47389007,-0.13637787,-0.13461097,1.3025367,-0.13763165,0.16016515,-0.09432387,-0.03235943,-0.2645581,-0.6840386,0.19629547,-0.5312021,-0.47319973,-1.3891981,0.86184233,0.21136567,-0.8533861,-0.64815027,-0.87334996,-0.92165405,-0.5408402,-0.26619357,-0.69230956,-0.44848266,0.7280645,-0.47514093,-0.7395029,0.5908054,-0.21804206,-0.13599822,0.4889344,0.30051097,0.3627466,-0.6264798,-0.2757047,-0.05244362,-0.42335054,0.44708833,-0.34365082,-0.6178576,0.06793687,-0.7347244,0.12991138,0.225275,-0.30854863,-0.63246816,-0.4336597,0.25024164,-0.42293236,0.4326635,0.39497417,-0.7206616,0.18624815,0.50268626,-0.7552665,-1.1931492,0.19575171,0.04530293,-0.15326332,0.5609911,-0.10186394,0.55084586,-0.25318268,-0.36266536,-0.5527624,-0.6328401,-0.3196206,-0.4653101,-0.07129173,-0.60346097,0.04418176,0.72503346,0.3716267,-0.7764507,0.07554694,-0.17630143,-0.17705797,-0.58358455,0.6388405,0.09628047,-0.74194235,-0.4385423,-0.0440093,-0.06275781,-0.10391887,0.49359947,-0.12642293,-0.28023884,0.30161634,0.12640662,0.1153751,-0.11559257,0.04418872,-0.58561575,0.0036359,0.7753954,0.08592941,0.49887523,0.21265374,0.3486298,0.12017833,-0.13866258,0.24680285,0.40073782,-0.410798,-0.02262405,0.10938445,-0.26858434,0.40944186,-0.6353954,-0.56728405,0.45082343,-0.21378066,-0.15403631,0.3538011,-0.12510374,0.0065373,0.8335986,-0.40123734,-0.17066908,-0.21418402,-0.4326129,0.34285632,0.18746164,-0.15312892,0.44421315,0.70516163,0.46411178,0.18193622,-0.08136763,0.03939817,-0.14963888,0.5982941,-0.2319247,-0.3518587,0.92045915,0.1029162,0.13943696,0.20411432,0.14571992,0.44958872,0.0878789,-0.72808635,0.5941134,-0.89529204],[-6.39288947e-02,-5.76802671e-01,4.67741579e-01,-4.13109928e-01,5.11524737e-01,3.98756236e-01,-3.08651894e-01,1.81865901e-01,-5.05335212e-01,6.08548075e-02,1.66912839e-01,-1.88663378e-01,8.36238205e-01,-3.13921243e-01,-3.71442258e-01,-5.47887236e-02,-6.28256083e-01,2.81747937e-01,2.78595872e-02,4.84472722e-01,3.64986122e-01,1.03853822e-01,5.38435578e-01,1.93786025e-01,-4.94424313e-01,3.56265217e-01,-2.79017370e-02,2.65748829e-01,1.59693629e-01,-6.16781354e-01,3.18638802e-01,7.43803978e-02,-1.37419090e-01,-5.06823421e-01,4.41819042e-01,-2.36561537e-01,1.33276835e-01,4.00255114e-01,6.59209341e-02,-3.52917492e-01,2.04014048e-01,-4.17643845e-01,-5.92222095e-01,5.08778393e-01,-1.60383672e-01,2.77701050e-01,9.38617706e-01,-5.12730181e-01,4.01136547e-01,-1.82126865e-01,-1.25422731e-01,1.39533415e-01,-7.90936410e-01,1.29611149e-01,-6.58780873e-01,3.40644717e-02,-1.84896469e-01,-7.35524118e-01,-7.36866653e-01,1.24741130e-01,6.34522438e-01,1.12318248e-01,2.91313887e-01,-2.60895193e-01,2.33582914e-01,-1.48775712e-01,-1.00735597e-01,-3.55621547e-01,-4.82877553e-01,5.96304238e-01,4.07435924e-01,7.62709677e-02,-3.92018706e-01,-1.10216939e-03,1.33976206e-01,3.72279137e-01,2.36094266e-01,-3.41477484e-01,3.42568755e-01,-3.52077633e-01,1.65125236e-01,-6.34235263e-01,-5.14005959e-01,5.54317534e-01,-3.65874350e-01,-7.73069501e-01,-1.44052684e-01,-5.12600243e-01,6.76445723e-01,-8.62757683e-01,-3.68774593e-01,-5.13119280e-01,-1.86514501e-02,3.19267929e-01,5.15523970e-01,-4.54390556e-01,-4.26417202e-01,1.32052321e-02,-5.41012287e-01,-2.80082464e-01,-1.66844577e-01,-3.45321596e-02,3.64621818e-01,-7.38149583e-01,2.21506283e-01,-4.52682115e-02,2.30799019e-01,3.67823541e-01,-4.96125221e-02,1.82487726e-01,-5.11154115e-01,-4.64567870e-01,2.70564437e-01,3.26277822e-01,1.75080523e-01,-2.06041913e-02,-4.19715792e-01,3.58405799e-01,5.15597798e-02,-4.39950526e-01,3.93158525e-01,-5.95905244e-01,-2.32356235e-01,-1.92671955e-01,1.64193273e-01,-2.57280767e-01,4.17706184e-02,-1.98739246e-01,-4.22419701e-03,9.99475002e-01,3.00253928e-01,-5.20319380e-02,4.93446738e-01,-1.85558051e-01,-2.95414776e-02,5.27630933e-02,-2.18373075e-01,-7.48730123e-01,1.25663370e-01,-1.53071746e-01,-2.90266573e-01,2.67699331e-01,-3.34590584e-01,3.46427590e-01,5.89838743e-01,-5.98874949e-02,7.08493412e-01,2.73096532e-01,8.42797849e-03,2.53408984e-03,1.17268436e-01,-1.08754838e+00,1.33230016e-01,3.79804410e-02,-9.29594159e-01,6.27831340e-01,8.90307873e-02,-5.83484650e-01,-1.13657951e+00,7.61304125e-02,6.12565994e-01,-1.41196281e-01,5.44565916e-01,2.81810343e-01,-6.64229572e-01,2.65779883e-01,6.51003718e-01,-1.10481814e-01,-3.73606771e-01,1.21038415e-01,6.53762281e-01,-4.31610011e-02,5.65714896e-01,-6.52946532e-01,-1.61985382e-01,-2.20773462e-02,4.93400097e-02,4.33945470e-02,7.26531088e-01,-3.56424212e-01,6.05702400e-01,-1.38193071e-01,1.55012891e-01,3.41484487e-01,-2.45091558e-01,9.01514366e-02,-2.14983672e-01,2.09686503e-01,-2.17763096e-01,-4.52557653e-02,5.65308869e-01,-2.39857674e-01,-5.03163040e-01,-1.49934518e+00,-1.06020164e+00,-3.34109426e-01,1.12307346e+00,-6.26144350e-01,-1.45015597e-01,3.53751600e-01,-3.28494087e-02,-2.80368537e-01,7.63814524e-02,5.31301379e-01,-5.57462096e-01,3.60162646e-01,-6.32334769e-01,3.80904347e-01,-1.20504610e-01,-3.14471811e-01,2.05522940e-01,1.27259806e-01,-3.89464766e-01,-2.58346856e-01,-7.93806732e-01,8.88960660e-02,-7.73473799e-01,-8.99899364e-01,3.45212251e-01,-8.63651693e-01,2.26113215e-01,-6.76266849e-03,-7.52321556e-02,-1.43400893e-01,9.75357890e-02,-9.46728051e-01,-4.18342531e-01,1.61639020e-01,1.22450620e-01,-3.27812620e-02,3.81716967e-01,-1.55029908e-01,-8.90499890e-01,1.52230948e-01,1.61230966e-01,-3.96878302e-01,-1.92426398e-01,-1.78754076e-01,-7.97379196e-01,3.56659628e-02],[0.45517483,0.2773049,-0.81190157,0.28688195,0.07800084,0.14553076,-0.12289548,0.30428866,-0.3281738,-0.6682811,-0.4914925,0.09541938,-0.08111633,0.5993207,0.4943166,0.1270626,0.45374054,-0.30688277,0.55123514,0.00904535,0.01983353,-0.13614748,0.01099644,-0.2789642,-0.01261291,0.2795666,-0.12288764,0.46125582,-1.0684392,-0.7010974,0.07540991,0.28476134,0.10025532,-0.2603729,1.1593974,-0.04958774,0.33289114,-0.30819622,-0.73652506,-0.2600478,-0.21796237,-0.71874875,0.01802398,-0.14423095,-0.6316538,-0.19295046,0.37136552,-0.21150397,0.07071327,-0.13665617,0.5633202,0.16106915,0.35233968,-0.1823836,0.01209788,-0.32793954,-0.25369883,0.5952486,0.5364129,-0.20147496,0.2775546,0.20217893,0.24419549,-0.0118527,-0.20748569,0.9455332,-0.27617502,0.29126072,-0.03061969,-0.5692458,0.1202544,0.84758544,-0.5880437,0.19988577,-0.26487255,0.05050159,-0.21934932,-0.19259018,0.2072649,0.28134307,0.4739937,-0.14093694,-0.51675695,-0.09268916,0.00473762,0.24972737,-0.50021935,-0.04344096,-0.4366156,-0.44193047,0.28124335,-0.14469196,0.6900007,0.00131329,0.13872813,0.17826541,-0.2485951,-0.21740086,-0.6015285,-0.14882368,0.01190766,0.56832916,-0.26681998,-0.22862017,0.61292,-0.28784308,-0.42231536,0.3681465,0.63267475,0.22187197,-1.1085232,0.0893743,-0.36189595,0.22817478,-0.16294023,-0.66060483,0.05918859,0.55950034,-0.22669698,0.01137114,0.5306076,-0.9502925,0.67305785,-0.38569736,0.54647845,0.314146,-0.14249122,-0.60299397,0.46119925,-0.30751878,0.23205091,-0.32609504,-0.3524242,-0.1712601,-0.6456709,0.22616538,0.20898755,-0.52310115,-0.11080348,0.1677231,-0.15435635,-0.00274736,0.28306308,-0.18008704,0.33079365,-0.41556397,-0.09345577,0.17720331,-0.22096075,-0.25459075,-0.22645605,0.2717914,-0.53290963,0.31913418,-0.23894344,0.4275049,0.60948455,-0.58174,-0.7352232,-0.53066164,0.19987974,-0.73086613,0.08621159,0.07866355,-0.37472644,-0.03921975,-0.13616697,-0.15875298,0.5231952,-0.38128123,0.42312235,-0.638271,-0.45392078,-0.57707936,-0.18045406,0.01726777,-0.2863828,0.04707926,-0.06551037,-0.2412349,-0.1130804,-0.07266923,-0.912302,-0.31631136,-0.07864524,0.38818,-0.1694831,0.00531621,0.814566,-0.13064225,0.25984892,-0.2704279,0.17346828,0.05789871,-0.2192025,-0.4032485,0.0095258,-0.7301872,0.48569953,-0.9874254,-0.27711812,0.2641693,0.29643995,-0.44544175,0.1330058,0.33637264,0.23860233,-0.307075,0.06997263,-0.36929005,0.29621753,-0.2374989,-0.00824513,-0.0628824,0.52930784,0.42969677,0.31059954,0.8952072,0.22492518,0.35446587,-0.3723866,-0.2162461,-0.48941517,0.3988391,0.6603757,0.05738009,0.11465108,0.19392553,0.7353811,-0.30620578,0.19517362,0.02956175,-0.43404472,0.19941783,0.33677173,0.11044321,-0.06264739,0.05057837,0.20769349,0.2790304],[0.1114863,0.26066345,-0.23649566,-0.11144283,0.04021421,0.12400639,0.24384144,0.05144868,-0.19604488,0.1341188,0.5144321,-0.46408,-0.24522096,0.3909946,0.11716702,0.3393852,0.5481732,-0.7158648,-0.71218085,-0.23698063,-0.29238582,-0.05552157,0.16372037,-0.0419188,-0.6138424,0.15090226,0.399285,-1.025504,-0.5722528,0.15117556,0.4258922,-0.0025863,0.8378983,-0.78974944,0.02905928,0.08089962,-0.3756808,0.21924306,-0.24030022,-0.9285756,0.608959,-0.24609609,0.42016125,-0.9879878,0.46752623,-0.1695194,0.01906649,0.05357867,0.4510396,0.12758002,0.15153426,0.08978532,-0.30735615,0.21097797,0.7628485,-0.6563745,0.20809008,-0.6504873,-0.39863443,-0.8637102,-0.42168993,0.57793725,-0.19719659,-0.26978758,-0.6814875,0.5495226,0.5959854,-0.2738915,-0.136949,0.31464803,-0.27208212,-0.67372066,0.02672627,-0.1421132,-0.1332016,-0.62576467,0.0510923,0.14134637,0.14032814,-0.66256726,-0.0228442,0.5731072,0.09228005,-0.73203224,0.03030497,-0.26977828,0.29745218,0.2382121,-0.13439335,-0.09681343,-0.25876796,0.51055574,-0.29170656,0.26601434,-0.65680426,-0.45126215,-0.21048328,0.14843412,0.5460908,0.12126812,-0.0587504,-0.4610761,-0.40258777,-0.19950038,-1.0771313,0.24570335,-0.27496263,-0.05493308,0.07431484,0.38164362,-0.16969489,0.23141117,0.17745182,0.16594963,-0.2602787,0.17151648,0.29464298,0.15641785,-0.5998752,-0.12966703,0.3015524,0.14829831,0.5349128,0.23221952,-0.04610958,0.28857687,-0.4270723,0.14325264,0.40316224,-0.06333238,-0.5111337,-0.10900064,-0.63268244,0.19811445,-0.48197004,-0.01693336,0.30211854,0.8632468,-0.4899827,-0.12797871,-0.96561915,-0.54559004,-0.10245013,0.22885934,0.06598057,-0.4727636,0.59666103,-0.43593308,-0.6026906,0.4900873,-0.32203558,-0.26214737,-0.3919711,0.14067487,-0.38297212,0.29240638,-0.66804236,0.4205481,-0.37697384,0.02931968,-0.08201925,0.3958575,-0.39200592,-0.37600696,-0.66408306,-0.15022172,-0.32665294,-0.00798097,0.46646464,-0.26185292,-0.1834839,0.32282722,0.5385374,0.25594446,0.16532782,0.2705007,0.02080354,0.4662416,0.6972052,0.32526514,-0.77690876,0.67824066,0.18424156,0.2198075,-0.03107001,0.001331,0.41044208,0.8453927,0.3275743,-0.18696488,0.02749508,-0.6580939,-0.2369271,0.22688344,0.8904666,-0.03493988,0.35786688,0.3657305,0.40525743,0.24538422,0.32249388,0.24483757,0.4315856,-0.5464147,0.20578127,-0.06605855,-0.40258723,-0.2741947,0.6050853,0.4748646,0.1727147,0.06497232,0.21028003,-0.11712094,-0.795453,-0.6413041,-0.39868245,0.05517691,-0.4531347,-0.16997664,0.0638987,0.47610196,0.6268868,-0.04620245,-0.6284687,0.11112438,0.2954575,1.0329808,0.29818282,0.5929648,0.20451264,0.07796944,0.52332973,-0.3102841,0.47637135,0.62264097,-0.63215435,-0.41554797,0.40523425,0.12781976],[-1.35866523e-01,3.18507671e-01,-3.59697342e-01,5.74625373e-01,-1.48355499e-01,-2.02924952e-01,1.06265575e-01,2.16408148e-01,4.48811084e-01,-1.61793306e-01,1.07325189e-01,2.37618253e-01,-2.10212562e-02,-4.35798407e-01,8.13445821e-02,5.43888390e-01,-1.64521337e-01,5.23836203e-02,-4.83450055e-01,2.17108801e-01,-7.51971543e-01,9.01093841e-01,-2.93389469e-01,1.92573622e-01,4.77667928e-01,-9.30873156e-02,5.17854512e-01,6.94690883e-01,-8.16265568e-02,1.37461042e-02,-1.62119567e-01,-5.07964969e-01,5.87447323e-02,3.94090146e-01,-2.19322935e-01,-4.29007769e-01,-1.10157318e-01,-5.34005798e-02,-1.29785061e-01,-5.83074391e-01,1.74831688e-01,-5.15492380e-01,4.09605116e-01,1.92958400e-01,-2.51660734e-01,9.30225030e-02,5.97216845e-01,-5.58256030e-01,-2.70202398e-01,-8.73919204e-03,5.73830120e-02,-1.39354125e-01,1.10546406e-03,3.28668535e-01,-1.30497843e-01,-3.91791850e-01,2.72963107e-01,2.84475889e-02,-1.64972574e-01,8.33716989e-03,-1.25213608e-01,5.20054281e-01,6.66281208e-02,2.87375063e-01,-3.92022759e-01,3.28207701e-01,7.88280591e-02,-2.02151716e-01,-5.16191721e-01,2.65974790e-01,1.11373745e-01,-7.26351321e-01,-3.54764044e-01,5.68603933e-01,-2.10880414e-01,-1.01903565e-01,-4.60124552e-01,-5.24647236e-02,1.25337735e-01,9.17058229e-01,9.05703604e-02,1.20489728e+00,-3.80248651e-02,5.00478260e-02,1.44230828e-01,-6.07177079e-01,-2.49232978e-01,4.07453567e-01,1.34778880e-02,-3.99647474e-01,9.78818014e-02,7.75812864e-02,-3.04964364e-01,1.21316388e-01,5.85317373e-01,-5.93593299e-01,-5.90184815e-02,-6.88993752e-01,-8.97445306e-02,-1.33137524e-01,-5.68448007e-01,3.22293311e-01,-5.03565848e-01,-7.62915984e-02,-2.35362016e-02,1.58931315e-01,1.34122580e-01,2.87323982e-01,-4.27343063e-02,3.02259833e-01,1.33711606e-01,2.11423635e-01,5.04607618e-01,-8.84628817e-02,9.47558433e-02,-3.53908390e-01,3.55185345e-02,-6.11518502e-01,4.17994618e-01,1.52789056e-01,2.04086781e+00,-3.32182676e-01,3.79254341e-01,4.61890221e-01,-4.83531177e-01,1.28013298e-01,3.05689573e-01,2.55918056e-01,-6.18071556e-01,1.77433282e-01,2.75472522e-01,4.05626088e-01,3.26586723e-01,3.20641965e-01,3.73311847e-01,-6.89930022e-02,4.08798218e-01,-1.16592310e-01,1.93822116e-01,1.40058100e-01,-9.55412835e-02,-9.16030779e-02,1.18080162e-01,-4.22663301e-01,-2.72415191e-01,-1.91266045e-01,3.14690880e-02,-1.73365042e-01,4.00696129e-01,3.42840433e-01,4.69812006e-01,-3.00755113e-01,2.17539929e-02,-3.33817303e-01,3.65418822e-01,-5.01455963e-01,9.05077934e-01,-1.83621824e-01,3.13922524e-01,-1.04621589e-01,2.50400990e-01,2.30515778e-01,8.20051193e-01,-1.84372038e-01,-1.15502357e-01,5.46184257e-02,1.41374528e-01,-3.40288341e-01,6.75354362e-01,-2.27917403e-01,1.39741629e-01,4.66083065e-02,-8.14847946e-02,-2.10119307e-01,4.82354194e-01,-2.93427557e-01,-7.49249995e-01,4.68771368e-01,-2.86506653e-01,-2.86778301e-01,2.73094565e-01,-1.99052900e-01,1.33849800e-01,-5.52783608e-01,2.59229183e-01,-2.14581475e-01,7.19049037e-01,7.60790706e-01,3.26673351e-02,8.14465106e-01,6.20783754e-02,7.16315359e-02,-3.09224933e-01,4.38572699e-03,9.28022504e-01,-1.19585924e-01,1.33149862e-01,-6.86461627e-02,1.52513102e-01,-2.53956020e-01,2.38281384e-01,1.70595929e-01,1.86031252e-01,-1.26051784e-01,-3.64147350e-02,1.33496150e-02,3.31165224e-01,9.09325927e-02,4.88917887e-01,-1.57401964e-01,2.17132822e-01,-5.27494967e-01,6.20199740e-01,-8.31399262e-01,-8.31200406e-02,-7.73899630e-02,7.48278320e-01,2.17557654e-01,7.11202770e-02,-1.98636234e-01,-4.75708805e-02,1.52177708e-02,-1.12532638e-01,5.88484593e-02,5.60263097e-01,2.92928338e-01,-1.52551234e-01,2.40740821e-01,-4.71205354e-01,5.45012616e-02,-6.06002808e-01,-1.84249550e-01,8.27025324e-02,8.09525430e-01,3.32924426e-01,-3.35091323e-01,2.87573040e-02,-3.45771939e-01,4.42930264e-03,4.90382344e-01],[-3.38097781e-01,-2.48840917e-03,1.34059429e-01,2.36639321e-01,-1.10895790e-01,2.47298434e-01,-5.97743765e-02,-5.93778431e-01,3.62917155e-01,5.61389387e-01,5.51955327e-02,-3.35505873e-01,-6.93459623e-03,-3.38153929e-01,4.54010703e-02,3.46888810e-01,-5.47590494e-01,-5.46502769e-01,1.36851382e+00,3.85722339e-01,-3.74123633e-01,-2.25032911e-01,4.58677322e-01,3.33586514e-01,-5.59403419e-01,2.64635503e-01,1.88608184e-01,1.99696183e-01,-3.60637397e-01,-3.22583526e-01,5.09159744e-01,1.66376024e-01,1.66018814e-01,-7.80523717e-02,3.01987231e-01,-3.68394196e-01,1.93564609e-01,1.54191151e-01,-1.07205540e-01,-2.51180500e-01,2.26158559e-01,2.66844928e-01,7.16365874e-02,3.37681949e-01,-1.21134765e-01,-1.22781277e-01,4.05884802e-01,7.50729620e-01,5.38988113e-01,-2.32592821e-01,-2.55077988e-01,-4.69984502e-01,4.15727645e-01,2.62707680e-01,-2.59385407e-01,-2.10201621e-01,1.43593684e-01,-3.91258895e-01,-7.00336635e-01,9.23181027e-02,-4.41530347e-01,-3.76594424e-01,-1.07790455e-01,2.00268719e-02,8.71667564e-02,1.31438404e-01,-4.59426612e-01,-4.43499207e-01,-2.85166711e-01,3.06901008e-01,-1.29419237e-01,2.09056422e-01,7.08833396e-01,2.12008640e-01,-4.96482998e-02,-1.93658203e-01,-1.36728927e-01,-6.67004645e-01,-4.36584800e-01,1.04137763e-01,4.70354438e-01,2.19917372e-01,1.46814883e-01,5.39751410e-01,-2.54873276e-01,5.53465784e-01,-1.20372832e-01,-4.38189246e-02,3.47134352e-01,-3.44169326e-02,-7.09651783e-02,-2.51641095e-01,-6.70467854e-01,-6.30496144e-01,-4.85407948e-01,-7.46031523e-01,1.88887399e-02,-3.87968361e-01,-1.52210519e-01,1.92269966e-01,-6.51534572e-02,6.45028651e-02,4.82617199e-01,-2.00263277e-01,5.15743256e-01,3.08120489e-01,3.65507841e-01,-1.84110373e-01,3.84735793e-01,1.42838672e-01,5.17372489e-02,-1.20319523e-01,-4.35446352e-01,-5.23643792e-01,-3.68177176e-01,-1.01401180e-01,1.35250047e-01,1.96509913e-01,2.52835751e-01,3.40905041e-01,4.06814754e-01,-1.22279689e-01,5.75337946e-01,-2.56041050e-01,1.06325589e-01,-5.46754062e-01,5.39056182e-01,1.41730413e-01,-3.80443871e-01,-7.01050520e-01,3.42367291e-01,3.97092462e-01,-4.58182782e-01,-7.92090952e-01,-8.19693625e-01,-4.05285269e-01,4.50689107e-01,-1.62886739e-01,1.63317770e-02,-5.35261109e-02,1.18401192e-01,-1.04687013e-01,5.29650092e-01,6.93574667e-01,3.38219613e-01,6.67354465e-01,-3.61685514e-01,-6.66687965e-01,-4.70440745e-01,-5.44352353e-01,9.98393516e-04,5.80249548e-01,3.30192894e-01,4.92505699e-01,3.15946370e-01,2.01064557e-01,3.25692713e-01,-1.86824709e-01,-4.87238653e-02,-1.68688633e-02,-1.97017491e-01,-4.93127912e-01,-3.69184613e-01,1.63853899e-01,-2.56828237e-02,7.22521484e-01,-3.07692170e-01,7.11622015e-02,-4.93531018e-01,-3.59297007e-01,-1.99427471e-01,-1.09649599e-01,-4.88534540e-01,-1.43552914e-01,-4.91004050e-01,-7.14254007e-02,-2.42834195e-01,3.00223142e-01,7.60843828e-02,-1.22220822e-01,3.34575400e-02,-2.39504904e-01,-1.04098523e+00,-8.57115388e-01,4.85302120e-01,6.37775242e-01,-2.52119362e-01,1.73921421e-01,1.36495441e-01,1.29461408e-01,-3.41915786e-02,3.96694571e-01,-1.74809564e-02,4.76219177e-01,4.40492719e-01,7.39056543e-02,1.14633597e-01,-6.39982102e-03,3.15429389e-01,-1.43662289e-01,2.17393383e-01,4.53294843e-01,-6.71397269e-01,-1.74662605e-01,2.80372471e-01,2.42949992e-01,-3.02985817e-01,-1.02395523e+00,7.53756106e-01,-1.69132233e-01,4.30023104e-01,-1.78173140e-01,-5.39465964e-01,-1.93851814e-01,2.45281190e-01,4.20079619e-01,-2.54804194e-01,-1.91520602e-01,-1.50633538e-02,9.72268060e-02,-9.38059986e-01,1.49162197e-02,-2.07688853e-01,-6.22321904e-01,1.24317840e-01,-3.94093454e-01,3.42786498e-02,-8.12995136e-01,3.75713468e-01,-7.95519173e-01,-5.38338423e-01,1.47367761e-01,9.60061997e-02,-6.40295625e-01,6.95111811e-01,4.21205521e-01,-6.56122863e-02,-6.71574175e-01,-4.10462856e-01,-6.11597180e-01],[0.18819109,-0.11755439,-0.07154546,0.04163655,0.345844,-0.01069411,-0.20431326,-0.07871872,0.5918542,0.20235544,0.21911833,-0.2031476,0.3027328,0.02023509,0.62312454,-0.12967844,-0.67100185,-0.06699176,0.13241078,0.57793975,0.3941737,-0.59219384,-1.0172912,0.29274118,0.6980124,-0.12254554,-0.12005559,-0.31989324,-0.56834894,0.57515323,-0.03260174,-0.30263418,-0.27060148,0.37217674,-0.15179873,-0.244795,0.13739632,0.15608683,-1.0326443,-0.30880085,0.6444746,-0.67429644,-0.08573256,-0.05792836,0.16786502,-0.4013044,0.2960715,0.03199017,-0.00384743,-0.14258245,0.09841093,-0.30284938,0.29436332,-0.6735519,-0.60951906,-0.15992007,-0.24327816,-0.34888732,-0.60303056,-0.40189078,0.10355914,-0.35183465,-0.38026214,-0.46113315,0.19761068,0.10948142,0.45101967,0.10042675,-0.427841,0.05049386,-0.348237,-0.18320723,-0.25305986,0.3277482,0.64989895,-0.30925912,0.53508735,-0.9839328,0.07841887,0.04538152,-0.15204236,0.04031322,0.58089864,0.11726674,0.36622757,0.45116445,0.4868298,-0.44949293,0.5378336,-0.13635755,-0.3267337,0.3743522,-0.39640236,-0.25612107,0.22121659,-0.4535998,0.06487677,0.10321511,-0.1538197,0.4073607,0.17060558,-0.18532316,-0.4777294,0.44971618,-0.05675548,-0.12408501,0.2613009,-0.614883,0.17059867,-0.18808974,0.6686664,0.16763042,-0.30828056,-0.34181964,0.15534432,0.00240916,-0.0635665,-0.23669535,-0.43688202,-0.42644727,0.11700963,-0.1909446,0.23903298,-0.13562724,0.3921312,-0.49222115,-0.17867585,0.91616684,-0.24168655,-0.02163169,0.04516814,0.17866756,0.17253846,-0.44396663,0.12426259,0.20212458,0.18661352,-0.22918986,0.23230946,0.22074713,-0.5211968,-0.26003605,0.46042866,0.08326819,0.23419607,0.7736991,-0.39693898,-0.2534519,0.06860562,0.12631495,0.5622744,-0.34956363,0.0729001,0.40527225,0.20231128,0.51505995,0.8597911,-0.10500938,-0.5017818,0.0054496,-0.3923115,0.03169752,0.118422,0.4901855,-0.34204426,-0.03976777,-0.8367466,-0.25781763,0.26306847,0.11708208,-0.2313596,0.4104955,0.09816692,-0.07284536,-0.22041349,0.15462309,-0.33574587,-0.21068165,-0.56151474,-0.02500366,-0.75649977,-0.46443185,-0.41849083,-0.283306,-0.02108973,0.10416527,-0.71255565,0.0210278,0.04092258,-0.86967313,0.07672258,-0.23292449,-0.46427897,0.18600796,-0.4630339,0.2298486,-0.02654459,0.13534927,0.1122867,-0.3402504,0.19347136,0.02805328,-0.00270296,0.6127501,0.29826602,0.03414558,-0.3497238,-0.00426872,-0.39081842,-0.3182121,0.09752654,-0.50410134,-0.2464308,0.4892268,0.7241469,-0.11990871,1.1380513,-0.4434148,0.28967616,-0.75970227,-0.21845978,-0.12789994,-0.15211612,-0.44156316,0.04860661,-0.327218,-0.48666716,-0.22927718,-0.65681183,0.1704534,-0.08366342,-0.19932176,-0.01189297,-0.43818778,0.28691593,0.06651424,-0.16159071,-0.26275924,0.23839127,-0.6096935],[-1.42978013e-01,-6.25748277e-01,-4.23961908e-01,5.23751557e-01,6.19289279e-01,3.49452972e-01,-4.79599446e-01,-7.73580521e-02,-4.60146606e-01,-4.31247130e-02,9.15486157e-01,-1.09071411e-01,-1.11176847e-02,-6.35383070e-01,-3.07106376e-01,4.64608639e-01,-2.06927225e-01,-2.93864995e-01,3.37107480e-02,3.98185730e-01,-9.19690728e-01,5.52014053e-01,-9.08359945e-01,-3.44923019e-01,3.01528245e-01,-5.57797849e-01,-5.51739410e-02,1.88337062e-02,2.55689591e-01,1.31520405e-01,4.23212767e-01,-4.00432765e-01,1.16590947e-01,3.12065870e-01,1.50805041e-01,4.98629421e-01,-4.87028152e-01,8.74187127e-02,3.85753781e-01,-2.73466051e-01,-1.86830133e-01,3.84108931e-01,8.39638989e-03,-1.34340763e-01,3.68920565e-01,6.27868831e-01,1.08323053e-01,2.48736441e-01,5.13367176e-01,-7.89004982e-01,-6.01098239e-01,2.98652887e-01,-3.66057187e-01,-3.93747419e-01,-1.09164310e+00,-5.31149330e-03,1.08206034e+00,3.43634516e-01,2.67020822e-01,6.08335435e-02,5.40445209e-01,-5.65862775e-01,3.17916155e-01,-1.45560622e-01,-2.73924351e-01,-8.26945305e-01,3.63857746e-01,-5.60766816e-01,-7.52479061e-02,-5.71095705e-01,5.06337702e-01,5.91194965e-02,2.44300947e-01,1.36702716e-01,-3.01681817e-01,9.93436575e-01,-7.01928973e-01,-3.51108819e-01,-5.55043101e-01,1.72066897e-01,-1.24095015e-01,2.90048718e-01,-3.18207085e-01,2.73400635e-01,-2.20521167e-01,-6.98609054e-01,-7.40212272e-04,1.12905644e-01,7.89623737e-01,1.17045775e-01,4.27445531e-01,3.87007654e-01,8.42904150e-02,-2.67028600e-01,-3.12673151e-01,-4.27926689e-01,-5.92945337e-01,3.87655824e-01,-1.71445042e-01,8.31342265e-02,4.99810547e-01,2.93053716e-01,-2.36748867e-02,-7.17562735e-02,-3.64684463e-01,1.12774894e-01,-7.68085659e-01,4.97509092e-01,3.35992038e-01,-1.20308436e-01,1.56383246e-01,1.33877993e-01,5.04888929e-02,-1.00438945e-01,-9.71913755e-01,6.41341090e-01,1.53467506e-01,3.05307716e-01,7.97313973e-02,1.75415158e-01,7.16235936e-01,5.34749031e-01,1.39159828e-01,4.06790107e-01,-1.10144682e-01,-1.83254749e-01,-2.30676964e-01,-4.81006950e-01,3.19371372e-01,-7.73279741e-02,-5.88152528e-01,5.79083741e-01,4.90504026e-01,-4.59896743e-01,1.93740681e-01,6.01125300e-01,-5.95325470e-01,4.06751603e-01,-2.53277123e-01,1.00462046e-02,-3.79828550e-02,7.51051605e-01,-5.76904528e-02,-2.00343132e-01,4.03809398e-01,4.84008342e-02,-3.05737883e-01,-3.88126150e-02,-4.61666621e-02,-3.00452471e-01,9.42726210e-02,5.40508449e-01,-6.35403171e-02,-3.28395039e-01,-2.64101505e-01,3.01058173e-01,4.14464712e-01,3.02966148e-01,-5.20225346e-01,3.02716374e-01,-4.47689146e-01,1.35281622e-01,6.59542382e-02,-3.77463818e-01,2.98733652e-01,8.96043032e-02,-7.42097348e-02,3.94106835e-01,1.55272707e-01,6.90842152e-01,-5.44617921e-02,4.13704783e-01,2.00895607e-01,-5.10245085e-01,-1.72043100e-01,-2.20751137e-01,-1.50847048e-01,3.22818279e-01,4.53583114e-02,1.31560355e-01,6.99590147e-01,-6.43351078e-02,-9.08185303e-01,-2.65066266e-01,-6.88131154e-01,-1.76177099e-01,-5.35712242e-01,-1.91830039e-01,-8.59122947e-02,-4.64278221e-01,-3.31514210e-01,7.01181870e-03,1.04153072e-02,2.21066073e-01,1.61069021e-01,7.88641751e-01,2.48551756e-01,-6.79190993e-01,3.11015815e-01,-2.15785965e-01,9.46363360e-02,2.33992800e-01,-2.90609211e-01,-1.52987674e-01,2.10095905e-02,2.24543676e-01,-2.12457672e-01,3.13530892e-01,-2.11785570e-01,-1.77898109e-01,4.39177811e-01,2.78550386e-01,-3.43715161e-01,-3.91821384e-01,1.72052518e-01,-3.01886946e-01,6.96055740e-02,-8.41714162e-03,-1.55304879e-01,-1.37819088e+00,1.01273626e-01,-5.11850297e-01,3.31260443e-01,1.62089556e-01,-6.42929599e-02,-9.89802703e-02,9.78182778e-02,1.34654164e-01,1.84550196e-01,-1.04159757e-01,1.18515372e-01,2.20700204e-01,-8.19722891e-01,4.28780198e-01,3.58304799e-01,1.73328504e-01,2.54289806e-01,-2.25493073e-01,1.00518787e+00,-2.24937588e-01],[5.08465350e-01,-3.11265856e-01,9.61388946e-02,7.17438683e-02,7.73250282e-01,-4.84437533e-02,-1.82200894e-01,3.16528261e-01,1.02998465e-01,5.27823448e-01,1.39287099e-01,2.97591120e-01,3.38866264e-01,-2.09752973e-02,-3.07410657e-01,-1.38885033e+00,1.85207665e-01,2.93061584e-01,-4.10504878e-01,2.05497429e-01,-2.01879650e-01,8.19459319e-01,-6.13783777e-01,1.10396457e+00,-5.28091848e-01,4.85911250e-01,-2.44185522e-01,-4.87154216e-01,-4.93672997e-01,-3.05706531e-01,4.61463451e-01,-3.88512343e-01,-4.90217954e-01,-1.79229453e-02,-3.96950722e-01,-1.94361225e-01,-4.27263170e-01,2.58700669e-01,6.48434367e-03,2.05423862e-01,-5.81031680e-01,-4.29105073e-01,-5.42494170e-02,-3.05405408e-01,5.64882159e-01,1.53315902e-01,-3.66360247e-02,-1.06543884e-01,1.07629662e-02,-1.17253594e-01,1.62624538e-01,-6.55895472e-02,4.02907938e-01,2.57345289e-02,-1.07477951e+00,9.65187013e-01,1.80030599e-01,-5.18363357e-01,-9.21605453e-02,-1.36278421e-01,3.57745618e-01,9.07585844e-02,-2.90516037e-02,6.64711818e-02,-2.50527263e-02,-1.17222778e-01,3.21994871e-01,1.25119925e-01,-4.54929829e-01,-7.45147243e-02,5.57979524e-01,-3.15788925e-01,1.85401291e-01,-1.19442217e-01,2.47935593e-01,1.22378856e-01,-4.93922085e-01,1.69283628e-01,-3.57670009e-01,8.54838490e-02,3.10523123e-01,4.21300232e-01,-1.40840992e-01,-4.47568268e-01,-3.69592309e-01,-6.70198023e-01,-7.95931369e-03,2.42039815e-01,3.25120017e-02,4.44181561e-02,4.00942154e-02,9.81185958e-02,-4.64568347e-01,1.90553382e-01,1.12633660e-01,-1.78690463e-01,4.26679313e-01,-2.92929530e-01,4.70434636e-01,1.18468769e-01,-9.01857913e-02,5.06944597e-01,2.36714885e-01,-1.21297151e-01,-2.08447367e-01,1.55644575e-02,-4.27815050e-01,3.55392486e-01,4.03242975e-01,9.10187140e-02,4.18441832e-01,-1.75086275e-01,2.23404288e-01,8.15819725e-02,4.04348642e-01,-3.11875939e-01,-2.03280777e-01,-5.99235177e-01,3.70579332e-01,3.38935331e-02,1.02405846e-01,-2.27575570e-01,4.41423029e-01,-1.78403163e-04,4.49863486e-02,2.13809192e-01,-2.34700486e-01,2.61116862e-01,3.38278383e-01,-3.84680271e-01,1.11248381e-01,1.70417577e-01,2.18049079e-01,3.31322342e-01,2.47621119e-01,7.08216608e-01,3.68696630e-01,-1.20337598e-01,-3.27530861e-01,2.32740827e-02,1.41353711e-01,1.77011006e-02,4.00948673e-01,-2.07097689e-03,-5.36071599e-01,-1.75439611e-01,4.02806401e-01,-1.30544975e-01,-2.46088654e-01,2.15730697e-01,-3.89043152e-01,8.71957932e-03,-7.01621890e-01,5.38646638e-01,-8.60325173e-02,-5.18252015e-01,3.17841649e-01,1.83578804e-01,2.11857006e-01,-1.99945942e-01,3.01288188e-01,-4.36869234e-01,1.71632409e-01,3.23297223e-03,-3.03958077e-03,-4.01206285e-01,-4.08450544e-01,1.99236333e-01,-3.50583076e-01,-4.10481423e-01,5.58624566e-01,-3.68624032e-01,-1.40185595e-01,-4.19746973e-02,-2.97615618e-01,1.21980466e-01,-1.31346273e+00,7.96080530e-01,-3.55183840e-01,7.04402253e-02,-2.76490897e-01,-1.99235633e-01,-5.73056579e-01,3.52496326e-01,3.50448132e-01,-3.02554637e-01,-1.10116951e-01,1.11079358e-01,5.06645620e-01,4.03821841e-02,2.02057213e-01,-5.83637774e-01,1.43225953e-01,-5.08433163e-01,9.54374909e-01,4.17046659e-02,1.26902655e-01,6.06469631e-01,-5.75652540e-01,7.68737137e-01,3.64251107e-01,8.07018518e-01,-1.01354145e-01,-4.46395397e-01,3.43952596e-01,2.97050267e-01,7.78398871e-01,-1.39278576e-01,-5.85939828e-03,3.04518074e-01,-2.75295198e-01,9.89854038e-02,-1.77496448e-01,-2.68332124e-01,-5.78132644e-02,-6.22288622e-02,7.97916353e-02,-6.97655320e-01,-2.56036550e-01,7.94344023e-02,-3.37018073e-01,2.77808905e-01,-7.13075995e-01,1.17125586e-01,7.09189475e-02,4.62591916e-01,2.69359589e-01,1.71795320e-02,-2.88254082e-01,1.70320213e-01,-8.30570608e-02,-1.84587300e-01,-3.12031806e-01,-9.89752915e-03,3.59055459e-01,-3.15813005e-01,4.27817613e-01,-3.22930455e-01,-1.32892340e-01,-2.28450224e-01],[-5.36673903e-01,-7.07943261e-01,1.99552447e-01,-4.89542633e-01,-2.72346735e-01,4.76697944e-02,2.32737541e-01,2.45660186e-01,4.80788678e-01,1.44741520e-01,-6.15550578e-01,3.32790524e-01,2.83667684e-01,2.01821119e-01,-2.13669851e-01,-3.86986613e-01,2.70612031e-01,-2.71615475e-01,1.83051392e-01,-5.16212583e-01,-2.38109648e-01,1.71570286e-01,4.19182390e-01,-3.67053926e-01,-3.14459726e-02,2.97046714e-02,4.26628292e-02,4.70804989e-01,4.71437544e-01,-6.21762872e-01,1.09493506e+00,-2.59979188e-01,3.86722177e-01,-4.60852593e-01,-2.27450237e-01,-2.17869133e-01,3.20268363e-01,5.51578045e-01,2.72998303e-01,-5.93638182e-01,2.80350596e-01,-2.63839681e-03,3.31365407e-01,-3.44477855e-02,-7.61090741e-02,2.19329596e-01,-4.50096846e-01,5.32446146e-01,8.26165497e-01,-3.42452705e-01,-5.31756222e-01,3.75100493e-01,2.16050714e-01,-5.72841406e-01,-4.98409361e-01,-8.93639699e-02,7.44546652e-01,-2.32787654e-01,1.61765948e-01,-4.72597219e-02,1.46847934e-01,3.55075240e-01,1.10670082e-01,1.45740779e-02,4.89571035e-01,-4.07475770e-01,-5.90267181e-01,-1.12585008e-01,-2.42682651e-01,1.19001918e-01,-2.51889408e-01,-2.44915605e-01,2.76641667e-01,-5.88710785e-01,5.03638685e-01,1.92278668e-01,2.65922815e-01,1.07132502e-01,-5.32049179e-01,2.29345277e-01,-1.62154585e-01,2.48480011e-02,-7.91185558e-01,1.85498416e-01,2.25178063e-01,-1.29082009e-01,7.11183026e-02,3.50025654e-01,3.69598061e-01,-1.79011017e-01,-6.79615438e-01,-3.63868743e-01,3.78888965e-01,2.93976724e-01,1.52430773e-01,5.15396833e-01,2.23694980e-01,2.52395589e-02,9.89815295e-01,-4.83173430e-01,5.01591861e-01,5.39903879e-01,-3.82327795e-01,-1.72664046e-01,-1.47090435e+00,-4.34168011e-01,-8.78035575e-02,-1.32869914e-01,-9.09690917e-01,-6.09985664e-02,-2.05566242e-01,-2.72095591e-01,-3.61202776e-01,-5.17041385e-01,-5.82777858e-01,-2.34195322e-01,-1.43411243e+00,-7.88926363e-01,4.32756282e-02,-2.08793744e-01,-1.41776521e-02,5.71820661e-02,5.13091564e-01,4.48594272e-01,1.38936177e-01,-4.37853038e-01,-1.99032295e-02,-2.39530176e-01,9.48990226e-01,-9.07209739e-02,-6.80199862e-01,-1.21124417e-01,4.45184648e-01,-2.07385018e-01,-3.50688964e-01,3.96145105e-01,-6.54667437e-01,2.13608891e-01,3.54156457e-02,-1.17665775e-01,-5.17347530e-02,2.07228232e-02,2.68309832e-01,-5.77408075e-01,5.56808054e-01,-1.87029570e-01,3.12528402e-01,-5.39991707e-02,2.13090897e-01,-4.01996702e-01,-3.47838283e-01,2.50147641e-01,2.37776041e-01,1.70539111e-01,1.98569506e-01,-4.85221088e-01,6.03839040e-01,5.12977898e-01,-3.81340891e-01,-5.84841929e-02,1.52743548e-01,2.13482499e-01,5.61897047e-02,6.84062302e-01,3.26238811e-01,-1.52314305e-01,5.09079695e-01,-5.57547212e-01,3.94754559e-01,-7.37114921e-02,-2.15941846e-01,-3.12604964e-01,7.36494184e-01,-4.66619372e-01,-1.02131271e+00,3.47833961e-01,7.95073956e-02,-7.93692827e-01,-5.00935316e-01,2.03005031e-01,4.86324847e-01,-5.35132706e-01,-8.22213471e-01,-2.04153657e-01,-4.26260948e-01,-4.87761915e-01,6.07237965e-02,-2.45433629e-01,5.75914383e-01,1.58805633e-04,4.98057067e-01,-1.74798355e-01,3.03842634e-01,-1.79355338e-01,8.22305322e-01,8.10034499e-02,2.29047045e-01,4.00038660e-01,7.11494163e-02,-1.37389392e-01,-8.26885626e-02,-6.14313245e-01,-9.09521818e-01,-2.30008319e-01,4.51529510e-02,5.73685586e-01,1.02720715e-01,4.14661527e-01,6.04780205e-02,-3.16321731e-01,1.30667508e-01,-6.66714609e-01,8.23978424e-01,8.81078601e-01,-2.98106223e-01,2.62409568e-01,-1.05143499e+00,3.28787386e-01,7.93560863e-01,-6.63522542e-01,-1.64242700e-01,-3.56720328e-01,-3.18977922e-01,2.55600750e-01,6.59827814e-02,4.06805217e-01,1.53059319e-01,-2.58661300e-01,1.19502179e-01,-2.08651900e-01,-6.48263574e-01,5.11528075e-01,-2.95877010e-01,3.15631628e-01,-1.07668257e+00,-3.99480984e-02,-2.17272982e-01,7.69567490e-01,-5.97833455e-01,-8.37283880e-02],[0.2979403,0.10555956,0.49391937,0.2818061,0.8947861,-0.3363275,-0.02571622,-0.20693527,0.31350762,-0.32565287,0.8557823,-0.9636389,0.7631179,0.34665814,1.2910773,-1.1175967,-0.49039927,-0.4556364,-0.20505378,-0.2928672,0.21281497,-0.24102099,-0.3160894,-0.3333739,-0.1307813,-0.100321,-0.03027,-0.18700533,-0.42771545,-0.3022848,0.4785265,-0.43426666,0.5308307,-0.8311342,-0.5659238,-0.0135056,-0.18330938,0.04359186,-0.27787757,0.47181374,0.3604524,0.2398909,-0.05795994,-0.28037444,0.26585266,-0.2529896,-0.52579993,0.43631846,-0.27125615,0.33754292,-0.02608026,-0.02748245,-0.18724161,-0.63339067,0.01680791,0.03125839,-0.00320709,-0.50073,-0.51675606,-0.21226865,-0.03599026,-0.1643122,0.03860743,0.5257302,-0.43325746,0.29854494,0.55733293,-0.05570117,0.0647549,0.54801494,0.28731784,0.19221652,-0.20970252,1.0006081,0.0948608,0.12197739,-0.30596936,0.11317155,-0.12689728,-0.27787662,-0.32928512,0.29683676,0.32668135,-0.02380514,0.06522527,-0.49038032,-0.21504635,-0.12424172,0.31251666,-0.05005377,-0.09329513,-0.07254531,-0.16303018,-0.11468774,-0.31349438,-0.35009858,-0.21942908,0.04028069,0.08252533,0.06388131,0.50906676,0.27838153,0.43349236,-0.35341284,0.76534706,0.00277203,-1.019073,-0.2254495,0.39650047,-0.15438564,0.32271525,-0.5909199,-0.16743892,0.46558785,-0.4328924,-0.34285715,-0.3905831,0.5207836,-0.9256353,0.21620806,-0.08270277,0.23622958,0.38860172,-0.23270138,0.57376623,0.05540585,0.5130787,0.07478155,-0.15688974,-0.19632079,0.36288956,-0.12615483,-0.00723232,-0.28898385,-0.3452701,-0.21776727,0.59732765,-0.12097494,-0.51491886,0.12814376,-0.31093547,0.04322667,-0.27049538,0.65034956,0.18929903,0.4049266,0.66289026,0.5293849,-0.05679909,0.72330654,-0.9135345,0.333879,-0.5291726,0.15861835,-0.23261021,-0.33419204,0.9629366,0.16999325,0.13631189,-0.38112935,0.01562559,0.39380923,0.2568708,-0.20972717,-0.13750659,0.37286595,0.39143732,-0.3803368,-0.10208326,0.4555125,-0.97547626,0.3902071,-0.47190624,0.5779763,-0.4122108,-0.5316698,0.01816255,-0.23078924,0.00716023,0.11786832,-0.25891802,0.02609904,0.2630363,0.58735216,-0.04447708,0.16907194,-0.29896548,-0.09651949,-0.45945916,0.27366024,-0.05630214,0.5863989,0.3489996,0.11940278,0.09096207,0.49391901,-0.26683408,-0.63280034,-0.10883251,0.18362746,-0.13215032,0.40589768,-0.2981814,-0.41717088,0.19334,0.38494155,-0.37000918,-0.277204,0.13941887,0.01377336,0.25251222,-0.22211681,0.05105744,-0.137215,0.5191075,-0.03143638,0.34965864,0.20108469,-0.569562,-0.16293664,-0.5049167,-0.18714455,-0.5591749,0.17811298,-0.410527,-1.4476256,-0.37955025,-0.5894019,0.18196498,-0.3582646,-0.6514206,-0.30990228,-0.0466077,-0.22479376,-0.38531446,-0.5206904,-0.13123661,0.36696878,0.40778372,0.0793669],[6.60824299e-01,1.92559898e-01,4.05680053e-02,-2.27309644e-01,-1.15708396e-01,-5.50922930e-01,-4.54137981e-01,2.73290724e-01,1.33266926e-01,-6.53620481e-01,5.73587477e-01,-4.84338790e-01,3.06215882e-01,-3.68594259e-01,4.40035433e-01,3.80410464e-03,2.42139682e-01,-3.08070689e-01,8.45582485e-02,1.93789020e-01,-2.13100031e-01,-4.09296870e-01,6.45027041e-01,4.24140215e-01,4.02516611e-02,-4.46117014e-01,-2.76714057e-01,-3.28002125e-01,-8.16968903e-02,2.19756156e-01,1.03900552e-01,-7.04652071e-01,7.11882263e-02,-1.21528828e+00,9.78353918e-02,-1.12062621e+00,1.76248714e-01,-6.46942854e-02,3.85930538e-02,2.24830404e-01,9.05200839e-01,6.53559640e-02,-9.37116668e-02,1.72114119e-01,-1.56011447e-01,-3.73156309e-01,-6.94459498e-01,7.47768044e-01,3.97461236e-01,-1.54574305e-01,2.19751045e-01,4.75618213e-01,-1.13411777e-01,-5.74123502e-01,-6.58490181e-01,4.84731406e-01,4.71820474e-01,2.31977981e-02,-3.67836580e-02,3.50639522e-01,-4.74956393e-01,2.83641338e-01,-2.10769728e-01,1.50852174e-01,3.21871579e-01,-5.11198580e-01,-2.22036529e+00,5.41663349e-01,-5.18854916e-01,-4.00273889e-01,5.44294640e-02,-1.46851301e-01,-5.34768939e-01,3.94988507e-02,-2.31679320e-01,-5.70320129e-01,-1.34936487e-02,1.46899819e-01,-3.60821672e-02,2.20252294e-02,3.39111179e-01,-6.56802297e-01,-4.12153751e-01,5.53416498e-02,2.27493674e-01,2.26847589e-01,3.00838381e-01,-3.72616947e-01,7.29520559e-01,-2.06240043e-01,-2.20737621e-01,1.30532175e-01,-5.50457686e-02,-3.86442840e-01,3.71862054e-01,-1.15719065e-01,-4.06178087e-01,5.36938727e-01,-3.73314053e-01,5.05694032e-01,-5.77682376e-01,6.92818016e-02,-1.18396811e-01,-3.05424005e-01,4.08915319e-02,4.05591130e-01,2.86305219e-01,-5.27077541e-02,3.58334512e-01,4.73330706e-01,-1.00446820e-01,-2.29333535e-01,1.80050105e-01,-5.25415778e-01,5.75888872e-01,-2.60664016e-01,-4.48020518e-01,3.30864400e-01,5.69021761e-01,3.30931872e-01,-4.56849933e-01,-5.74814528e-02,-6.83124661e-02,-1.12443782e-01,6.73447549e-02,-2.58143574e-01,2.61736333e-01,-5.74012756e-01,-8.08209404e-02,1.65240709e-02,6.97738603e-02,-2.10629895e-01,1.04685508e-01,-2.60611415e-01,8.23690444e-02,1.19120687e-01,-4.83671129e-01,6.64973080e-01,2.78642982e-01,6.35800242e-01,-3.46684873e-01,4.22800332e-01,-6.42390668e-01,4.91411030e-01,-5.28516233e-01,2.24155858e-01,-3.05738837e-01,-2.53934473e-01,2.33239621e-01,1.09984139e-02,7.00599313e-01,-3.20312828e-01,5.58220983e-01,5.54113425e-02,4.72867787e-01,-1.47992402e-01,3.13415825e-01,4.28605020e-01,1.11310923e+00,-4.99895334e-01,-2.02768639e-01,-4.09405679e-01,6.75096586e-02,3.26255828e-01,-3.51783633e-01,-4.47758019e-01,3.50715548e-01,4.93072644e-02,4.31525886e-01,-6.62825167e-01,1.37572959e-01,-4.13543075e-01,-2.33274400e-01,4.70516413e-01,4.17500079e-01,-6.30964711e-02,1.09177935e+00,-3.49388868e-01,-2.92134583e-01,-1.59120886e-03,9.98483300e-02,4.47210193e-01,-1.55877620e-01,-2.76714750e-02,-1.05865307e-01,-5.06161690e-01,-3.98311377e-01,2.93283492e-01,-2.98848718e-01,5.78127682e-01,9.51748490e-01,-1.37854889e-01,1.79447815e-01,1.85447633e-01,5.17640635e-02,-9.27461237e-02,-8.01701903e-01,8.33146870e-01,-5.16258478e-01,2.33644336e-01,-1.18928544e-01,2.02856600e-01,-4.21035200e-01,-5.37983656e-01,4.01567519e-01,2.39087760e-01,2.94815481e-01,-2.68777668e-01,-7.07658157e-02,-8.93623233e-02,5.61664224e-01,-1.50053054e-01,-9.67025697e-01,2.96997100e-01,-1.96401179e-01,1.18036106e-01,2.20909029e-01,-6.36123419e-01,-1.30695939e-01,-3.70789856e-01,-2.18683966e-02,-1.28281087e-01,-7.83549249e-02,-1.16313688e-01,3.01613539e-01,5.05510211e-01,8.40833262e-02,3.97830188e-01,4.07799751e-01,-1.79905444e-01,-7.11400926e-01,5.33819385e-02,5.00270545e-01,-5.64697862e-01,5.89619637e-01,-1.94058642e-01,7.68736839e-01,-5.43312013e-01,-8.31658486e-04,1.63445726e-01],[-5.50586954e-02,-5.23966610e-01,-4.51125383e-01,-2.71630473e-02,-3.26821506e-01,-3.37005883e-01,2.31084868e-01,-6.99997783e-01,-2.25739524e-01,8.15641165e-01,-1.25919029e-01,1.40141830e-01,4.05439615e-01,6.90085948e-01,-2.36465693e-01,2.19383970e-01,-1.03123002e-01,-1.31046474e-01,-8.82174313e-01,-7.36768186e-01,2.04060245e-02,3.30054238e-02,-1.41133487e-01,-4.74761069e-01,-5.98788142e-01,-2.64875859e-01,-1.05624065e-01,-6.53587654e-02,3.27677697e-01,3.18751931e-02,-3.72402593e-02,-7.16519594e-01,-6.43012941e-01,5.25501609e-01,-1.51941432e-02,-4.12588149e-01,2.87188649e-01,-8.42915997e-02,-2.05057729e-02,2.08056122e-01,1.46460444e-01,-1.71221018e-01,-6.56124055e-02,-4.19431746e-01,-9.01126415e-02,7.07955718e-01,5.25413334e-01,-4.19096261e-01,-9.85295355e-01,3.94567311e-01,-4.70041424e-01,-5.48885822e-01,1.49251506e-01,2.31164411e-01,-1.09423809e-01,-3.93581510e-01,-1.04419863e+00,4.22402829e-01,-9.68449056e-01,2.30746195e-02,2.51982540e-01,-6.34443834e-02,-5.95604554e-02,2.13112459e-01,2.48309761e-01,-1.54154599e-01,-1.72302723e-01,1.48864150e-01,5.28932959e-02,-3.51264834e-01,4.45828199e-01,-8.38079810e-01,-1.33761317e-01,4.64527428e-01,5.11794209e-01,-8.78187269e-02,-4.32974964e-01,-7.47118831e-01,5.85249551e-02,-1.06762163e-01,-1.16786696e-01,4.90990847e-01,7.44209051e-01,-3.31038713e-01,3.44105691e-01,-6.35774016e-01,8.63287300e-02,-4.89785850e-01,-8.60097647e-01,1.48380985e-02,-7.16065109e-01,-2.07441568e-01,-2.58706450e-01,1.98142111e-01,-1.06840909e+00,2.37461910e-01,-3.45656216e-01,-6.05930686e-01,1.25414625e-01,8.39248657e-01,2.46310890e-01,-1.90300927e-01,-7.24773228e-01,3.75578612e-01,-7.02170253e-01,-9.24346149e-01,-6.13973379e-01,2.20313802e-01,3.88838425e-02,-5.89205384e-01,-6.89112246e-01,5.13487533e-02,-2.68892080e-01,-5.02277970e-01,-4.99045551e-01,-5.45556068e-01,7.85167441e-02,-3.63178313e-01,1.07554100e-01,2.72392213e-01,1.63108874e-02,-6.36591375e-01,-2.14449182e-01,3.74072939e-01,8.35148022e-02,7.04140812e-02,7.94781029e-01,1.51626453e-01,-8.09825778e-01,-3.49113405e-01,-4.98746544e-01,4.10422534e-01,1.46658793e-01,8.10777128e-01,4.52290356e-01,-5.59502602e-01,-1.10902481e-01,-5.99901736e-01,6.68138079e-03,1.62017152e-01,4.60065268e-02,3.27399820e-01,1.41474307e-01,-3.48524779e-01,-1.32811293e-01,7.15304554e-01,-2.02418581e-01,3.21763575e-01,7.37790346e-01,6.07444704e-01,-2.37321988e-01,-1.23431838e+00,-4.38847244e-01,1.10052064e-01,-5.06185591e-01,1.99880347e-01,-1.79143503e-01,1.02965377e-01,-5.89588940e-01,-3.39234024e-01,-5.39637208e-01,-1.06705880e+00,4.57850844e-01,7.07104564e-01,-5.89678809e-02,8.48157644e-01,-9.53741521e-02,2.89087921e-01,-7.07810283e-01,2.79060811e-01,1.28154948e-01,-6.06059134e-01,-6.24076054e-02,4.41712409e-01,-9.46217030e-02,4.09198254e-01,-9.20419097e-02,-1.07361227e-01,-2.89202809e-01,1.93837732e-01,4.27890927e-01,9.48876202e-01,-1.72505528e-01,6.43655181e-01,-1.08626127e-01,2.54674017e-01,-5.44723086e-02,3.63720953e-01,6.25184504e-04,-2.44723144e-03,1.00752547e-01,4.70573872e-01,-7.37676695e-02,6.79836094e-01,3.87674928e-01,-3.24705511e-01,4.77060437e-01,-5.07164262e-02,2.05987185e-01,4.09863032e-02,-2.11527973e-01,7.11802363e-01,-2.73268893e-02,1.53262727e-02,-1.46167532e-01,1.15748778e-01,8.41844559e-01,-6.59130037e-01,1.28462970e-01,1.43273205e-01,-2.12756202e-01,2.07761422e-01,-7.16700971e-01,-3.96393031e-01,6.71539187e-01,-4.31585848e-01,-5.20132482e-02,-2.50704773e-02,2.64615566e-01,1.35584608e-01,9.16411281e-02,-7.64487982e-01,5.51287055e-01,3.96559209e-01,9.09991041e-02,-1.22564480e-01,1.84671611e-01,-1.02045834e+00,-3.48386774e-03,-7.09666967e-01,2.33861580e-01,-2.59485953e-02,-5.60029805e-01,5.34053445e-01,-8.63887072e-02,1.95536360e-01,2.58501679e-01,-4.60309312e-02,-3.43362331e-01,-1.89903051e-01],[-0.17182529,0.20157978,-0.14710163,-1.279281,-0.46637177,-0.12509553,-0.14952804,-0.43899676,-0.20191662,-0.29540387,0.5649656,-0.46321553,0.06107523,-0.48792735,0.40400997,0.5636512,-0.19156152,0.24370548,0.02683493,0.14537717,0.12139937,-0.21262181,0.47580665,-1.9559089,-0.24963176,-0.12203918,-0.36693126,0.42137042,0.08446199,-0.09254207,0.39612994,0.10980643,0.05889502,0.41986337,0.2453442,-0.39531413,-0.09702259,0.25790906,0.18974704,0.67066437,-0.85430586,0.13885768,0.512719,-0.16321373,-0.02541475,0.0527318,-0.16126993,0.09434724,0.19276884,-0.5154343,-0.29637158,-0.4070742,0.2648609,-0.08572856,0.3220499,-0.8551584,0.36411813,-0.5890417,0.11838829,-0.21173252,0.2857878,0.06527761,-0.35654274,0.54579973,0.03359859,-0.60727763,-0.31133032,-0.03371798,-0.41613632,0.1737328,0.4914633,0.40439397,0.47487003,-1.0690296,0.2971995,0.03791735,0.7202463,0.57194436,0.20697087,-1.3105597,0.11242267,0.7582091,0.27962595,-0.15963538,-0.33236262,-0.2952883,-1.592495,0.39325514,0.25092152,0.7634096,-0.03141011,0.2466555,0.02456209,0.02779078,-0.06128429,-0.32431045,-0.73458654,0.2103497,-0.40996847,-0.22891305,-0.05907159,-0.12901376,-0.3524539,-0.59258664,-0.2291839,0.21119174,0.08522639,-0.0646701,0.392528,-0.46219796,-0.03012305,-0.22759688,-0.41980678,0.6895326,0.0390097,-0.5872965,-0.13040242,0.5241584,-0.5754091,0.5814027,-0.3179987,0.12309369,-0.13215184,0.4313348,-0.30728474,-0.630133,-0.14144105,-1.3337512,0.51859826,-0.17184064,-0.31705907,-0.1536594,0.5803445,-0.0965049,-0.09468602,0.7927757,-0.28435495,-0.339046,-0.42494303,-0.2217875,-0.534986,0.26121148,-0.0564369,0.2567312,-0.62972647,-0.68140763,-0.18856488,0.660267,-0.15297714,-0.24381101,0.44662517,-0.98283356,-0.22150008,0.226696,-0.465564,0.7485944,0.16037795,0.08130696,-0.61767745,-0.03009428,-0.8069781,-0.4144608,-0.09905674,-0.37691894,0.08092131,0.0932066,-0.08958314,0.02892604,0.32662994,-0.73697674,-0.7357886,0.24089342,-0.6141689,0.08718041,-0.19016588,-0.10033081,0.01499243,0.7849101,-0.61237913,-0.316654,-0.22438613,-0.29169446,0.04816115,0.34188062,0.08559508,-0.8710887,0.18435453,0.01087096,0.78030604,-1.7581295,0.30014768,-0.9834181,-0.19354388,-0.23777214,-0.06264053,0.7127495,0.02454816,0.18539727,-0.53433216,0.3845067,-0.1735649,-0.7226779,0.5567776,-0.78832555,0.5376688,-0.33075204,0.68984306,0.34480694,-0.57547355,-0.23812014,-0.3773112,-1.2576683,1.0288947,0.46496993,-0.48545492,-0.5044665,0.5199076,-0.54701185,-0.28877535,0.10828945,-0.55869216,-0.26442003,0.04846334,-0.25507608,-0.69947094,0.2587806,-0.5586717,0.02640902,0.03837373,0.03021145,0.34662944,0.26906925,-0.2418206,-0.14041354,-0.20695557,-0.08502699,0.35951942,-0.17806499,-0.317846,0.37799025],[2.90764660e-01,4.93821949e-01,3.73431027e-01,7.85725415e-01,-2.93528363e-02,5.78400418e-02,8.36878359e-01,-8.02605808e-01,6.23421311e-01,-7.59218812e-01,7.20186353e-01,-6.72110260e-01,1.65018067e-01,-5.37865341e-01,-2.14206457e-01,-1.16355568e-01,-2.49799162e-01,-2.07581744e-01,7.21623063e-01,-2.14578107e-01,-1.14492267e-01,6.07182980e-01,-4.24688578e-01,-8.18837285e-01,1.08083077e-01,-4.91216153e-01,-1.17751077e-01,-2.44600937e-01,4.09710675e-01,-2.08560973e-01,-7.46850908e-01,-3.83850098e-01,5.50635643e-02,4.56004560e-01,-9.98815149e-02,-3.07282418e-01,1.44231066e-01,-5.22761531e-02,8.31660867e-01,-6.68030349e-04,-9.66386423e-02,-2.58177996e-01,-2.40102023e-01,1.80428237e-01,-7.45364606e-01,3.93264294e-02,-1.02567160e+00,-4.20830280e-01,5.91010988e-01,6.31051809e-02,-7.08811164e-01,-9.62557316e-01,-5.53265139e-02,3.75298798e-01,2.90493876e-01,4.57840919e-01,2.21782252e-01,-1.75304681e-01,-1.30613908e-01,1.42837569e-01,-3.47449899e-01,-3.46708894e-01,5.97074807e-01,-2.25742739e-02,-2.59125113e-01,-1.54795302e-02,-5.60043991e-01,6.91633284e-01,-3.91200513e-01,-7.86448047e-02,-7.73508102e-02,-2.75782943e-01,2.19778284e-01,5.31503916e-01,-2.48379752e-01,6.97899938e-01,1.85649276e-01,-1.19404957e-01,-4.97960709e-02,7.01700985e-01,4.58708137e-01,2.38671973e-01,-1.79236114e-01,1.13161817e-01,-2.76711851e-01,2.98560075e-02,1.84216812e-01,8.04142505e-02,-1.24022365e-01,5.76282620e-01,1.79721579e-01,-4.25578445e-01,-2.91967005e-01,8.48711506e-02,-1.15608305e-01,6.21744944e-03,-4.18977529e-01,4.59842682e-01,-3.76491435e-02,-2.13936925e-01,-1.53926134e-01,2.59376734e-01,-6.93206966e-01,3.02285135e-01,-2.17521042e-01,-4.03407097e-01,2.66395241e-01,-4.95095700e-01,4.07585591e-01,3.87352884e-01,4.14172620e-01,2.04250172e-01,2.60958612e-01,-3.55738178e-02,-4.93647940e-02,4.54705395e-02,-5.09327292e-01,1.27210066e-01,5.52031040e-01,-3.24339300e-01,-2.60309219e-01,-7.68207788e-01,-7.26130158e-02,2.78857172e-01,-5.23619235e-01,-7.48354793e-02,3.44327629e-01,-6.28218651e-01,-1.52316719e-01,-6.93705678e-01,2.27309763e-01,-5.50762475e-01,-2.97032833e-01,-1.00643778e+00,-1.02738492e-01,2.26077642e-02,-3.86238247e-01,4.62841034e-01,4.81112689e-01,-8.56564790e-02,-2.20674857e-01,-4.62145388e-01,-6.61702633e-01,-2.44568542e-01,5.54321766e-01,5.65418825e-02,-1.95827872e-01,-5.38827002e-01,-8.64098728e-01,-1.64833277e-01,-1.31723080e-02,9.80908051e-03,4.22750205e-01,-2.54332095e-01,-1.97975561e-01,3.45047534e-01,1.21523030e-02,-8.66571665e-02,-4.11249429e-01,-2.50028670e-01,-3.81185114e-01,3.78138721e-01,-7.14710355e-02,4.54210281e-01,7.52710879e-01,3.01790331e-02,2.76907176e-01,-7.09590092e-02,3.25982690e-01,2.54675269e-01,5.58012053e-02,-1.28346458e-01,1.75851434e-01,-2.42354020e-01,-1.87832832e-01,-7.06446946e-01,-2.72361577e-01,5.48328400e-01,-5.43068796e-02,-2.06576094e-01,-1.81043334e-02,3.48779202e-01,-2.10203081e-02,5.93688250e-01,1.89287011e-02,5.82959354e-01,4.57154185e-01,-5.19689620e-01,5.77497959e-01,3.27997118e-01,3.55425954e-01,-1.15564883e-01,-6.86187565e-01,-1.18433666e+00,-5.88253677e-01,-4.77390662e-02,-1.14120029e-01,-2.29804918e-01,1.71742812e-01,-5.52962184e-01,-6.08356237e-01,1.10049076e-01,8.50689560e-02,3.07260677e-02,-3.67276996e-01,4.65595797e-02,-3.31188500e-01,5.31549990e-01,1.92947924e-01,-2.44064387e-02,-1.63615972e-01,4.15327221e-01,3.13194633e-01,-1.49709001e-01,-6.29156977e-02,-1.44047454e-01,4.52109754e-01,-9.89677727e-01,-2.39355460e-01,7.14295730e-03,-7.20668375e-01,-1.43105045e-01,-1.18433237e+00,-4.48789269e-01,-4.61067736e-01,1.32501245e-01,6.65291190e-01,-2.93243289e-01,5.89341640e-01,4.64753387e-03,-3.40291679e-01,-5.70163071e-01,-7.79541612e-01,-5.37516534e-01,8.69473696e-01,-1.75298452e-01,-5.10090351e-01,2.75793165e-01,-1.54001832e-01,5.53576887e-01],[0.25350866,-0.14356725,0.07149969,-0.1878664,-0.36770514,-0.0637304,-0.39330536,-0.2166747,0.6229707,-0.02246646,-0.3011468,0.21304648,0.07254947,0.37587613,-0.57635003,-0.07849223,-0.30506086,-0.6898363,-0.47880393,-0.8923475,-0.16266462,-0.13594618,-0.39612535,0.09126042,0.59118736,-0.10024471,-0.5274483,0.4036183,0.16322596,-0.45188183,-0.42504528,0.00500645,-0.33914152,-0.29713967,1.3810843,0.43332148,0.40684468,0.07036744,-0.26188356,-1.1720568,-0.01219611,0.65166855,0.14190288,-0.41933426,-0.600435,0.90184027,0.16616336,0.22941674,-0.67999655,-0.81053126,-0.12635572,-0.18433437,-0.1009694,-0.09201276,-0.24839385,0.3340559,0.4720537,0.12323663,1.0608069,0.43346673,-0.5431092,-0.15380567,-0.45063454,-0.34849787,-1.0920649,0.8635468,0.44200194,0.0778274,-0.8779621,-0.38756385,0.7631728,0.3754288,-0.4367549,-0.11729822,0.66967386,0.565994,-0.14905706,0.0196108,-0.7090526,0.90455675,0.7147518,0.09583678,-0.3856651,0.3677081,-0.43488878,0.0581568,0.24183041,-0.24236108,-0.203154,-0.9447356,0.19772694,0.2650755,-1.0386759,-0.6782266,-0.170406,-0.05496213,-0.04806747,-0.02663424,0.03990891,-0.41454762,0.7463947,-0.12695241,-0.41836458,-0.18471648,-0.02776281,-0.2361979,-0.31347746,0.5685696,1.163116,-0.64714956,0.09233032,-0.32453272,-0.41218525,0.04866864,0.7211949,-0.21781059,0.21693663,-0.4963867,0.27181357,-1.1024318,0.06113593,-0.13365598,0.4438215,0.13664258,0.39250034,0.02468037,-1.0640876,-0.3814629,-0.1778363,-0.53688914,-0.27731004,1.089729,-0.19878423,-0.3309808,-0.39086947,0.17025048,-0.07835618,0.1625743,-0.6489013,-0.5884864,-0.07865077,-0.16043586,0.56256187,-0.00723859,-0.07451846,-0.38385865,-0.22923078,0.3345132,-0.55077,-0.27566367,0.02608149,0.84577954,0.36280614,-0.5878467,-0.89823705,-0.21683724,-0.16906534,0.1318493,0.01541455,0.22864851,0.02472062,0.45413655,0.3800465,-0.11597149,0.05434406,-0.88260853,-0.51781255,-0.26455763,-0.01994115,-0.8855583,-0.76279914,-0.002856,-0.24491054,-0.1234802,-0.02230434,-0.59525704,0.26611748,-0.43193173,0.41719684,-0.6456172,0.16898273,0.03068354,0.6719194,0.1416904,-0.5089763,0.2843251,-0.90571177,1.1436801,0.6249233,0.17743936,1.2260557,-0.05750809,-0.28391922,-0.47784445,0.17922156,-0.19846083,-0.02785598,0.04538803,0.06866604,0.37403882,0.03663504,0.2401533,0.14559773,1.2172163,0.14411665,-0.47786754,0.29191667,0.6466301,0.31573814,0.09646603,-0.25685507,-0.24389455,0.5341118,0.1355025,-0.57271874,-0.17130813,0.0493409,-0.11400689,0.14595327,0.16127495,0.22968698,-0.47619212,-0.02846482,-0.1055176,0.01772361,0.7229707,0.53463006,-0.38463852,-0.17585033,-0.5999654,-0.25960004,0.43064556,0.5188988,0.27452913,-0.46487045,-0.2360937,0.18436909,-0.51171166,-0.23211934,0.38763642],[4.82338905e-01,-1.62853524e-01,-7.09314525e-01,-4.72955145e-02,-1.24163009e-01,-3.37512046e-01,-5.73263764e-01,-8.26918185e-01,-8.04558694e-01,4.34479028e-01,4.38420653e-01,2.64994102e-04,-3.79065096e-01,1.95196252e-02,4.49734539e-01,-3.92615765e-01,-7.38424063e-02,-8.61664772e-01,-1.95922583e-01,1.32488877e-01,-3.25648308e-01,-2.39111278e-02,-3.41233909e-01,-4.78531539e-01,-1.70658779e+00,-9.02793288e-01,-6.26203477e-01,-9.34975445e-01,6.98873028e-02,1.57160193e-01,-1.00758240e-01,4.01283085e-01,9.97637808e-02,9.63324606e-02,-3.16011369e-01,3.12258769e-02,1.98332801e-01,-2.96411626e-02,4.04094934e-01,-3.88255358e-01,-5.04596889e-01,-4.72759247e-01,1.71209887e-01,-4.04542774e-01,2.52124608e-01,2.90569723e-01,1.24101825e-01,2.85721570e-02,2.14561984e-01,1.07970305e-01,-7.36032486e-01,2.96116233e-01,-1.20572761e-01,-7.16518052e-03,-2.12852899e-02,-2.52588928e-01,1.02505434e+00,-5.79434156e-01,7.13184714e-01,4.13408786e-01,-1.49838656e-01,1.01816528e-01,-1.74339399e-01,-1.13911033e-01,3.50960493e-01,-4.27222103e-01,2.47941315e-01,-4.81373459e-01,8.54822934e-01,3.11543196e-01,-4.96048868e-01,1.01621822e-01,5.27895510e-01,1.07589535e-01,-2.46439874e-02,2.18374386e-01,-5.38733780e-01,-9.30055499e-01,-5.65703750e-01,-8.01219344e-01,-1.80840939e-01,-5.89536965e-01,6.93178535e-01,-5.50945580e-01,-1.64193049e-01,-8.40897143e-01,6.26232028e-02,4.40105826e-01,-2.47840285e-01,2.15597913e-01,-4.92099039e-02,6.74648404e-01,-5.63888431e-01,-6.79086670e-02,6.45253658e-02,2.15427518e-01,-1.90743059e-01,-2.34931409e-01,-4.69038278e-01,3.16943824e-01,-2.24954989e-02,-8.54592144e-01,1.37850299e-01,1.05259493e-01,-5.90234518e-01,-9.85773951e-02,-4.76318955e-01,-2.75564015e-01,-1.68001449e+00,-1.05694413e+00,4.17952925e-01,-3.99103582e-01,3.69918466e-01,-5.84752440e-01,2.58947045e-01,2.79243410e-01,1.49124768e-02,-4.03391629e-01,-1.00948918e+00,8.09051991e-01,8.28904659e-03,1.62776291e-01,-1.15499318e+00,2.40346000e-01,-2.53032167e-02,-4.10102457e-01,4.93494242e-01,-6.94499791e-01,-1.61088899e-01,2.23528102e-01,-7.77695417e-01,2.59296119e-01,-1.19729567e+00,-5.87137759e-01,2.77237713e-01,8.87349471e-02,-3.26926559e-01,-7.22381175e-01,-2.35207930e-01,-5.81205070e-01,4.39968169e-01,-6.41603649e-01,-7.75693059e-01,5.09120803e-03,1.33028612e-01,-2.36148030e-01,6.03977501e-01,2.27600560e-01,-7.50190318e-02,-5.19928694e-01,1.65688097e-01,3.61932814e-01,-1.38887703e-01,3.24236006e-01,-2.34081775e-01,3.14911723e-01,-5.47391437e-02,-2.26827905e-01,3.01005989e-02,-1.52419046e-01,1.29299140e+00,-4.40071113e-02,1.83613867e-01,1.78890765e-01,-3.51100326e-01,6.30849525e-02,6.12919092e-01,2.57939458e-01,-7.79925942e-01,1.79973498e-01,-5.02986908e-01,2.58939862e-01,-3.97602409e-01,-1.22089446e+00,1.18804052e-01,1.07834511e-01,-2.28039492e-02,2.37477556e-01,-4.09298003e-01,1.53745949e-01,8.54786634e-01,-1.03642568e-01,4.63090122e-01,-4.31634067e-03,3.38781208e-01,-3.53341252e-01,-1.25071526e+00,4.88604099e-01,-3.31479251e-01,3.77122939e-01,-1.16993856e+00,3.50527018e-01,-7.91998327e-01,4.64062579e-02,-3.42900634e-01,-7.80973375e-01,-4.02141690e-01,2.92220771e-01,-4.69643325e-01,-1.06281258e-01,-1.25119030e-01,7.66307209e-03,1.21226676e-01,2.57227998e-02,-4.69921231e-01,-1.25206396e-01,4.44293052e-01,-3.39844793e-01,-2.19972149e-01,-7.16345370e-01,-9.99239624e-01,6.97474658e-01,-2.76283085e-01,-5.11563361e-01,-2.35801131e-01,5.39976656e-02,4.01494047e-03,-2.62240004e-02,-3.24711651e-01,-3.46763372e-01,6.52908027e-01,8.41830820e-02,4.84070390e-01,-4.31644265e-03,-2.18880130e-04,-2.09959358e-01,3.63444984e-01,2.47915313e-01,-3.48197579e-01,-4.28146362e-01,2.80031532e-01,2.61040092e-01,5.94175100e-01,-2.38200173e-01,3.30590069e-01,-4.44944352e-01,9.99440134e-01,-2.59663165e-01,-1.58753717e+00,-8.46241117e-02],[4.44856286e-01,-2.14322999e-01,1.00132294e-01,-3.93952370e-01,1.73569664e-01,2.37639342e-02,-7.37261623e-02,2.71645367e-01,-3.40292335e-01,-5.99849880e-01,1.09571135e+00,-3.96077216e-01,3.01833361e-01,3.28276634e-01,8.37854385e-01,5.25450289e-01,-2.90266275e-01,3.67295235e-01,-1.17310151e-01,1.76426694e-02,2.41309553e-01,-6.42821789e-01,-2.99359769e-01,-1.70132458e-01,3.08115125e-01,-4.70168859e-01,-2.52994388e-01,-3.08916986e-01,-1.32696822e-01,2.18475871e-02,-8.27764332e-01,3.72493528e-02,-1.23612925e-01,4.89359528e-01,-1.56287819e-01,-3.55747402e-01,3.61538291e-01,9.06924009e-02,-2.27369666e-01,1.81074291e-01,-8.72400045e-01,-8.23306013e-03,3.91745120e-01,-1.36236832e-01,-1.79039836e-01,5.26961505e-01,1.95837617e-01,5.21402001e-01,3.57719034e-01,-3.07351947e-01,2.04995684e-02,2.35686749e-01,-8.44452754e-02,4.34172362e-01,-4.47404563e-01,-2.07100749e-01,-1.50988609e-01,4.81536090e-02,2.24190950e-01,-6.24024756e-02,-2.71825165e-01,-2.41489038e-01,-5.89728773e-01,1.23912826e-01,-3.41188192e-01,-2.13820308e-01,5.40595293e-01,5.04676521e-01,-4.53092456e-02,4.16179113e-02,4.80547965e-01,7.70724714e-01,3.06966573e-01,3.03497255e-01,5.64072967e-01,2.68840432e-01,-7.52868950e-02,4.38525796e-01,-5.02221584e-01,4.38054204e-01,8.89542758e-01,-6.06692612e-01,-4.06971931e-01,-6.71113253e-01,-7.26093531e-01,5.12357987e-02,3.16697478e-01,3.87607932e-01,5.41826606e-01,-1.21819280e-01,-6.66386867e-03,1.98072314e-01,2.04278633e-01,-5.70438385e-01,-1.76528648e-01,3.02846789e-01,9.84875951e-03,4.20776755e-01,8.35668072e-02,-1.14277817e-01,6.09762669e-01,-4.97103184e-01,1.62503660e-01,-9.20630038e-01,2.84217745e-01,1.81149855e-01,-1.00383162e-01,2.03905061e-01,8.37263837e-02,-3.57480258e-01,-2.10609972e-01,-1.42492667e-01,-3.08666348e-01,3.97243708e-01,-6.80518985e-01,-6.15915433e-02,-3.44532758e-01,-4.49188799e-01,4.90557030e-02,-3.91034603e-01,-8.04103494e-01,5.19607186e-01,-1.13175482e-01,5.40553749e-01,-3.28232437e-01,-9.93864775e-01,2.09489375e-01,-8.56607735e-01,2.05815449e-01,2.19097242e-01,4.11389679e-01,-6.25196457e-01,1.35558739e-01,5.57929993e-01,2.93054163e-01,-1.73921779e-01,-1.12801218e+00,5.20498753e-01,-4.88189757e-01,7.03031430e-03,2.27837861e-01,-4.05315310e-02,-6.97788954e-01,9.75087658e-02,-8.48457456e-01,1.75082758e-01,7.95600004e-04,6.34487122e-02,-1.15758073e+00,-1.47279724e-01,-1.56549364e-02,6.89313933e-02,-4.28988457e-01,1.79740474e-01,9.04965818e-01,-4.65429157e-01,-2.35398695e-01,6.97754085e-01,-2.58916527e-01,-4.44852829e-01,-2.18001716e-02,-8.73973608e-01,6.20055571e-02,-2.71953866e-02,-5.20386517e-01,-1.72193810e-01,-3.60475741e-02,-1.92247584e-01,2.87389047e-02,1.32905856e-01,-1.09151220e+00,-1.47415310e-01,-6.39103055e-01,4.61316437e-01,6.02127872e-02,2.34115720e-02,4.51955318e-01,-7.45065212e-01,-1.16883270e-01,-5.82199514e-01,-1.02770023e-01,1.63036287e-01,4.37872767e-01,7.73845464e-02,7.06532970e-02,3.10618162e-01,-4.21768546e-01,8.51912498e-02,-1.85652509e-01,9.17632133e-02,-6.21873498e-01,-1.49247676e-01,-3.27765226e-01,5.83639503e-01,2.61108041e-01,-4.96547312e-01,-2.96557665e-01,-2.25498080e-02,-1.68798789e-01,-6.95351958e-01,2.61481971e-01,-1.10239911e+00,-4.67383951e-01,-8.67745042e-01,6.36563241e-01,-1.90403149e-01,-5.87463304e-02,1.17055051e-01,3.39434803e-01,5.84967494e-01,-6.34367406e-01,2.65253246e-01,2.54954964e-01,8.09993565e-01,-2.52455883e-02,-3.73131990e-01,-5.56953490e-01,1.17179774e-01,-6.19690061e-01,-1.58494368e-01,-7.58166462e-02,2.71825701e-01,-5.66323586e-02,-1.40493318e-01,-1.28389382e+00,-3.80306295e-03,2.97665566e-01,6.18347466e-01,3.14502835e-01,4.09622639e-01,6.12098396e-01,-2.63769716e-01,6.47783875e-01,-2.34729256e-02,5.98311126e-01,-6.33740067e-01,1.10950254e-01,3.07279110e-01,8.37878436e-02,-7.69314528e-01],[0.02755515,-0.19877812,0.14230815,-0.13358174,0.07616656,-0.8561407,0.13178332,-0.0897243,-0.02724594,0.77057153,-0.17255068,0.04012,-0.37606382,-0.21225962,-0.03145269,-0.599651,0.2283883,0.4263863,-0.19859697,-0.2710313,0.7177073,0.51438344,0.45926192,0.2361991,0.11510832,0.25567317,0.36947304,-0.03955037,0.07674738,0.25355405,-0.35297155,0.5314885,0.20910563,-0.1368629,-0.90819687,1.2050004,-0.5177538,-0.15540941,-0.2157851,-0.37398952,0.05089682,-0.20347308,-0.01818373,-0.51241684,0.08036629,-0.6828603,-0.20352231,0.5052885,-0.18514144,-0.32291192,-0.04202303,0.6288047,0.11597098,0.61873597,-0.00551089,-0.07578398,-0.063885,-0.00646667,-0.36739156,-0.4743513,0.06523862,0.07741996,0.3061672,-0.2060075,-0.0397059,0.07255799,0.04194684,-0.19048645,0.23825923,-0.20837662,-0.5153416,-0.7911381,0.3175173,2.1629744,0.07168366,-0.06458683,0.20533392,0.63171345,0.61718404,0.915989,0.6287986,-0.33927768,-0.26904696,0.42478874,-0.33618078,-0.10561673,1.0333,0.12784001,0.22367088,-0.24268107,-0.2159745,-0.25576735,-0.19297695,-0.24319017,-0.21474093,-0.46347553,-0.17564258,-0.25295645,0.09070772,0.6078921,-0.01914406,-0.6791638,0.04159318,-0.1389715,-0.05837022,0.59010893,-0.4517621,0.32381603,0.270314,0.3024113,-0.11578297,0.11163436,-0.04575777,-0.3926154,-0.51026434,-0.56943333,-0.19665252,-0.15900594,-0.15665215,-0.01568977,-0.05942159,0.4545558,-0.21160918,-0.4569779,0.21309164,0.10883647,0.22536969,-0.14381689,0.24215952,-0.06986535,0.40097335,0.8478088,0.09984443,0.06190282,0.1040915,0.35296378,-0.15621953,0.3280046,0.04260678,0.06286837,-0.13383013,0.2727822,-0.12072939,0.19869217,-0.24835157,0.21277966,0.09906671,-0.03541509,-0.17006947,-0.03715517,-0.39593986,-0.18762395,0.04429229,0.17754221,-0.6363099,-0.29704332,0.45946303,0.08645933,0.12726544,0.49638513,0.390407,-0.8500081,-0.28678375,0.4093845,0.9161336,-0.41668737,-0.04422092,-0.3679015,0.533089,-0.4858702,0.43294916,0.13655399,0.03951624,-0.25375345,-0.08011341,0.19659287,0.36603865,-0.6367421,-0.09752358,0.48744878,-0.27232927,0.60808134,-0.09419597,-0.45744872,-0.11180889,-0.05264569,0.16806866,-0.41241103,-0.24888527,0.69159085,-0.1805221,0.16704674,-0.03182743,0.04454142,-0.35459477,0.26146662,0.47170165,-0.19134901,0.5312243,-0.15210702,0.29006833,0.401552,0.2891401,-1.2931832,0.39641958,0.10293975,-0.13951667,-0.11310949,0.5074487,-0.03485777,-0.23407103,0.18434021,-0.16513194,-0.06761633,0.15780802,0.22003256,-0.5957784,-0.25036684,-0.14881735,-0.37541792,0.12798561,0.10382376,0.10730101,-0.11332782,0.27036396,-0.43288794,-0.01306815,0.3843967,0.32224345,-0.4090481,0.2635955,0.24463503,-0.19832379,0.27819428,-0.2190126,0.5925256,-0.04479776,0.03958559,0.11382018,0.03603896],[2.23951891e-01,1.85921993e-02,-9.13744807e-01,-4.33790125e-02,-6.04915172e-02,-6.05409384e-01,3.13572407e-01,6.91001654e-01,-3.71417224e-01,2.92528272e-01,-3.81554008e-01,1.09074585e-01,-7.10735559e-01,-3.17367315e-01,-1.86512440e-01,-3.12906325e-01,1.31275311e-01,-2.73948431e-01,-6.06691360e-01,4.02388215e-01,2.04538256e-01,-5.44957060e-04,2.05933861e-02,3.44708115e-02,2.81195164e-01,2.27048337e-01,1.01942629e-01,-5.03185332e-01,8.69403005e-01,-4.91068393e-01,2.93915957e-01,-6.14856303e-01,1.40190810e-01,1.03921212e-01,-2.69710273e-01,1.17916666e-01,-2.95790043e-02,2.49095395e-01,4.15846378e-01,-1.08835295e-01,5.25647998e-01,2.31819719e-01,2.72316754e-01,1.72052905e-01,2.31403977e-01,-2.98278183e-01,-6.19609118e-01,2.83173412e-01,2.16800734e-01,-1.54290767e-02,-2.50520974e-01,-1.64659873e-01,3.90763432e-01,1.54531583e-01,1.02928019e+00,-6.42463565e-01,-6.11573279e-01,-5.78666572e-03,8.88409615e-02,-2.72270024e-01,-5.40684819e-01,-9.85136509e-01,5.26407421e-01,-6.37347639e-01,-1.53406397e-01,4.32053924e-01,6.15581572e-01,2.04794500e-02,1.57554090e-01,1.73438773e-01,-2.65186727e-01,8.97447467e-02,4.64655049e-02,-2.79618174e-01,-3.55593674e-02,1.92254007e-01,8.10031295e-01,-3.42378050e-01,2.12998465e-01,1.30075589e-01,-4.69669372e-01,3.17437589e-01,-8.22757408e-02,1.10297225e-01,-3.82012993e-01,-5.42428255e-01,-3.68549079e-01,-1.83456182e-01,-3.10318112e-01,2.93069541e-01,-2.65875489e-01,3.23613912e-01,1.64915025e-02,-4.18692753e-02,-7.56372154e-01,-2.71925747e-01,1.74567372e-01,3.80461782e-01,7.49294341e-01,3.31325054e-01,-5.69994636e-02,3.27556103e-01,6.30493015e-02,2.69130498e-01,2.96155572e-01,-6.79507293e-03,1.18199058e-01,-5.57719409e-01,-1.22379407e-01,-2.20471844e-02,-3.90887856e-01,-1.38524875e-01,4.05520946e-02,-4.84649613e-02,6.54924102e-03,-1.53282434e-01,-1.41801655e-01,5.40853739e-01,2.35674843e-01,-1.03121527e-01,-8.65215838e-01,-3.36820702e-03,-2.62746155e-01,9.29950237e-01,2.64218599e-01,5.52348733e-01,9.60800573e-02,-8.84940177e-02,6.66683689e-02,-2.59700775e-01,6.83508217e-02,6.19917989e-01,3.42807949e-01,1.94162115e-01,-2.21473962e-01,-4.71887052e-01,3.52301091e-01,-5.02655447e-01,4.27579992e-02,1.40030682e-01,-6.37650371e-01,-1.73340678e-01,6.85430020e-02,-6.42802238e-01,-5.75357862e-02,9.00762379e-01,-7.37143159e-01,-4.78022277e-01,-3.43252532e-02,-9.72380459e-01,8.16817343e-01,-1.06095672e-01,6.49525642e-01,1.14257382e-02,5.65167665e-01,-3.13497245e-01,-9.23642367e-02,-4.83566336e-02,5.83804607e-01,1.83914885e-01,-1.97820216e-01,2.70977736e-01,-5.30526876e-01,-7.22920418e-01,4.30740505e-01,-4.65986729e-01,-1.70788735e-01,-7.09542751e-01,-6.93607807e-01,1.59568593e-01,-1.01796780e-02,-2.67215788e-01,-1.44909501e-01,1.30783528e-01,-2.34274179e-01,1.93377152e-01,1.96279123e-01,5.09475231e-01,-1.14939697e-01,-9.38157260e-01,-4.00879562e-01,1.82791978e-01,-7.97951639e-01,3.37697417e-01,-5.05183995e-01,-3.96562308e-01,4.28937346e-01,-5.70403636e-01,6.55302703e-02,1.75237224e-01,-2.69114655e-02,7.77806640e-01,-7.20720664e-02,-1.11015216e-01,5.44694901e-01,5.54525912e-01,5.03551602e-01,-2.21491888e-01,5.76705217e-01,-1.03141502e-01,1.74196914e-01,1.96095601e-01,6.24281824e-01,-2.24288300e-01,5.17021894e-01,8.50018561e-01,-8.43879342e-01,-4.40973550e-01,-2.58700818e-01,7.04068691e-03,-6.24902606e-01,3.12179357e-01,1.65487472e-02,3.52489889e-01,-8.28072429e-01,-1.80754751e-01,-2.36451641e-01,1.67605087e-01,7.50622332e-01,-3.51749718e-01,-6.27941564e-02,-1.19047342e-02,2.49180809e-01,2.16015369e-01,-1.70964465e-01,7.29246214e-02,3.36409003e-01,8.66905332e-01,3.53573650e-01,1.84542686e-01,1.37305975e-01,2.91342676e-01,2.83752143e-01,-7.64270186e-01,-6.43044651e-01,-1.14790905e+00,1.67963177e-03,-3.63601625e-01,3.25710058e-01,8.69103611e-01],[4.69699621e-01,3.89348298e-01,1.51990633e-02,-5.62179387e-01,-4.66247573e-02,4.66239333e-01,1.78777754e-01,-2.28082612e-01,-9.45216656e-01,4.42745201e-02,-1.62264556e-01,1.43721804e-01,-5.51336944e-01,5.87397099e-01,2.13637277e-01,5.44836879e-01,-4.36653167e-01,-2.53600068e-02,-3.09513807e-01,-2.92106837e-01,-1.25914800e+00,-9.84137133e-02,5.96590757e-01,3.70889413e-03,-1.35364461e+00,-3.39993060e-01,-1.17140591e+00,-3.50883347e-03,-3.68089944e-01,-6.97059706e-02,3.05955485e-02,2.00455531e-01,5.92717588e-01,-1.08735240e+00,8.89661849e-01,4.08079743e-01,-1.49094298e-01,-2.76495069e-01,1.70219809e-01,4.35090393e-01,5.26724398e-01,-1.72671482e-01,-2.62580037e-01,-4.92839396e-01,1.04307377e+00,8.71590793e-01,-1.43569037e-01,-2.00063184e-01,-5.06848037e-01,1.98827181e-02,-8.76402929e-02,-6.68740094e-01,6.21342808e-02,-4.93243009e-01,2.45464966e-01,-2.11161450e-01,-9.44932774e-02,7.60929465e-01,2.04828247e-01,9.76158142e-01,2.82208342e-02,1.29953250e-01,6.45803571e-01,7.31265023e-02,-1.05510819e+00,-4.03106183e-01,9.08388123e-02,-2.91783810e-01,-2.20215440e-01,4.38876003e-01,2.73867369e-01,2.95737833e-01,-1.40720392e-02,4.48172033e-01,8.86545718e-01,5.20611823e-01,3.45907770e-02,-2.14753181e-01,6.24958098e-01,-1.86600733e+00,2.21639842e-01,-5.13297796e-01,1.72410980e-01,-1.54710516e-01,-9.71899927e-01,-5.78931332e-01,-8.41633618e-01,-1.78977549e-01,-4.40918326e-01,-1.16085596e-01,5.10110110e-02,-1.12704061e-01,-5.62695026e-01,-3.65715206e-01,3.85273099e-02,4.99891043e-01,-4.09260482e-01,6.38288319e-01,-3.28522325e-01,-3.45310211e-01,2.83219159e-01,2.64557987e-01,-1.90179765e-01,-1.90279931e-02,-1.17696452e+00,-3.40213865e-01,3.96405458e-01,9.52080786e-02,-6.58328354e-01,-6.59927607e-01,-2.08381429e-01,-4.24750410e-02,3.97823691e-01,6.31803036e-01,-2.37074018e-01,5.67123890e-01,3.68319154e-01,5.80608606e-01,-1.48608446e-01,7.43591972e-03,2.87859201e-01,2.41180182e-01,2.45435033e-02,-3.04292887e-01,-1.00702137e-01,-1.34153932e-01,-7.25815818e-02,-3.42649341e-01,4.42903638e-01,-4.50066090e-01,2.98581943e-02,-8.81543934e-01,-3.21002513e-01,4.15224791e-01,6.82964623e-02,-9.88193303e-02,-6.95694208e-01,-2.01549798e-01,-3.30947608e-01,-3.73374254e-01,6.31229579e-01,-3.82479757e-01,-3.50975394e-01,-7.34304607e-01,-7.55827129e-01,-5.03353715e-01,4.04152989e-01,4.73432690e-01,-6.52915597e-01,-9.51672718e-02,-5.16475085e-03,-8.45178783e-01,-1.17348611e-01,-1.11183679e+00,8.41585696e-02,-5.32449484e-01,-3.67078520e-02,1.70343325e-01,5.92879020e-02,-7.84347281e-02,1.01214677e-01,1.43396065e-01,2.26214841e-01,-1.47180513e-01,1.87291488e-01,-1.92097262e-01,-1.76700562e-01,-9.22307670e-02,3.40374351e-01,-6.92993820e-01,7.60126561e-02,-3.84424299e-01,-1.29376099e-01,-3.55190754e-01,6.00912273e-01,-6.42418444e-01,2.67987072e-01,-1.17732864e-03,-7.31920063e-01,-5.46244420e-02,5.51547110e-01,3.86443019e-01,-1.31007480e-02,7.23467290e-01,2.04762205e-01,1.49380118e-01,3.89643580e-01,-4.09396589e-01,5.04538596e-01,1.44805506e-01,5.06308228e-02,2.68243253e-02,-4.15865093e-01,3.60245645e-01,-5.09763002e-01,-8.71128798e-01,-1.41844094e-01,-5.96016645e-01,5.89725852e-01,1.51917890e-01,2.11475462e-01,-1.17328651e-01,-3.01175356e-01,-5.55669725e-01,2.52735359e-03,-1.15719306e+00,6.72035664e-02,4.96008471e-02,-4.97747362e-01,-3.78590554e-01,2.52807409e-01,4.22255754e-01,2.01576322e-01,4.10709791e-02,-2.39174664e-01,1.02268986e-01,-4.15081143e-01,-1.20965265e-01,8.82300436e-02,-7.97913194e-01,2.10980818e-01,7.06217349e-01,1.78345650e-01,2.78386146e-01,3.44470710e-01,-3.58287424e-01,-3.66793834e-02,-2.14855552e-01,4.26503867e-01,1.82058409e-01,6.10326231e-01,2.12706581e-01,-4.36627269e-01,4.63153332e-01,-3.61643106e-01,-8.69453847e-01,9.55252945e-01,-4.58612204e-01,-7.03396916e-01,-5.31125605e-01],[0.016731,-0.56218153,-0.03589086,-0.45790723,0.06711849,0.3555433,-0.18426819,0.29033634,-0.10693768,0.5694204,0.5459399,0.13826331,-0.3215178,0.5080629,-0.43589664,-0.56003255,-0.59852576,0.3973779,-0.264383,0.24445654,0.23326305,-0.39206094,-0.17781073,-0.36740085,-0.08292621,-0.58029264,-0.231142,-0.09501523,0.04293417,0.19469643,0.0772221,-0.19258583,0.60960513,0.31387588,0.83099014,0.11561576,-0.26248643,-0.05602023,-0.42388335,0.29849464,0.1532681,-0.30308917,0.43107337,0.50154173,0.18570921,0.7985852,-0.2512686,0.92579657,0.37357718,-0.24287738,-0.68965685,-0.23206986,-0.38502312,-0.1978675,0.04893743,0.21952789,-0.5413642,0.1058885,-0.42625064,0.0058076,-0.457748,0.68908453,-0.4130552,0.05854236,0.14441401,-0.29169214,-0.00618055,-0.01304968,0.14972393,-0.17157955,0.1566418,0.34698224,0.31065112,0.24801709,0.16043282,0.30398223,-0.27084482,-0.04550614,-0.25015992,-0.31213966,0.22667693,-0.22849397,-0.6921113,-0.84046316,0.10985927,0.10463186,0.28462845,-0.01752475,-0.7731307,0.38814685,1.0465451,0.14117567,0.3244883,-0.14568537,0.9683012,-0.7112964,-0.6513374,0.4256903,-0.41062528,0.17712077,0.56031996,-0.43899655,-0.04102869,0.12390887,-0.11004797,0.45799536,0.14245209,-0.5120899,0.18409961,-0.21405132,0.23164926,-0.2087579,-0.68252075,-0.52011347,-0.28767592,-0.3729187,0.11457073,-0.46533176,-0.7359021,0.21230748,0.13437316,0.453455,-0.02350594,0.34641096,0.09053186,-0.15542771,-0.23720373,0.46895266,-0.06413417,-0.33917066,0.15218537,-0.6997947,-0.7995541,0.06627402,-0.01911141,-0.27930674,-0.45089957,0.34522387,-0.26902074,0.758747,-0.0668993,-1.1489872,0.6402174,0.0698593,0.50747776,0.37631485,0.31268656,-0.02964178,0.17233962,-0.333968,0.25421798,-0.6328713,0.16126348,0.26649204,-0.55236757,0.62333846,0.16053547,-0.606787,0.00834752,-0.11171289,0.38756105,-0.647613,0.5927099,0.21968213,-0.03070017,-0.08192112,0.10074315,-0.09433906,0.08050088,-0.08457127,0.24359716,0.59848434,-0.42531267,-0.26289254,0.5007096,-0.5198427,0.35551676,-0.5890086,-0.16704473,0.23539008,-0.03653493,-0.47551656,-0.19890156,-0.62092674,0.3822923,0.00917613,0.05630748,0.4456966,-0.55060023,0.20847835,0.03253248,-0.4680625,0.38614604,-0.37034276,0.42509937,1.1470112,0.46080375,0.22435579,0.32077202,-0.12329378,0.6619878,0.46224478,0.3382874,-0.18431166,-0.13966104,-0.10472989,0.38734663,0.4704749,0.07170352,0.74968076,0.5054575,-0.2046202,-0.03173383,-0.04510807,-0.7868521,-0.05138875,-0.22181886,0.00780574,-0.21913765,-0.21170639,0.01282377,-0.2659125,-0.06713606,-0.76554894,0.37137997,0.24755344,0.23156789,0.4508117,-0.5036574,-0.16011855,0.04908682,0.4251351,0.3219314,0.78262323,0.42806813,-0.39521536,-0.5059222,-0.80524606,-0.07843097,-0.70486253],[0.03810196,-0.11895412,-0.62806493,0.11171191,-0.27983844,0.04017299,0.2145781,-0.29643384,0.06125459,0.07304021,0.41914922,-0.16135581,-0.06936451,0.33704424,0.5816723,-0.73807037,-0.1279975,-0.49904403,-0.12863424,-0.33514863,-0.30600893,-0.17627715,0.3086524,0.48763952,-0.0948155,0.22065039,0.16690277,0.2308986,-0.41456088,-0.4663909,0.11359029,0.22783221,0.50588673,-0.17733054,0.3048375,0.38806307,-0.23637985,0.14907122,-0.3123454,0.33013538,-0.15043746,-0.84591854,-1.0873278,0.18522716,0.32098043,-0.43216327,-0.2348399,-0.5021512,0.11251075,0.6626468,0.12233469,-0.30370003,-0.18668294,0.7596327,0.37203473,-0.04914037,0.20908651,0.30614698,0.38464636,0.3937872,-0.12014437,-0.5081373,-0.9820197,0.11144979,-0.15510342,0.2838273,0.33508477,-0.26598728,0.28404543,-0.15637334,-0.3214368,0.5318761,-0.48338544,-0.88892686,0.28568703,-1.4289781,0.05122475,0.08892712,-0.07186954,-0.25293627,-0.2553733,-0.06229346,0.116657,0.06945741,-0.66985255,-0.21324694,0.07306104,0.5801612,0.52348876,0.11998084,0.4429354,-0.39890715,0.17685066,0.61025727,0.04922944,0.5590022,0.4081514,-0.69372314,0.19771351,-0.17157581,0.10303681,-0.15177304,0.43833718,-0.08712342,0.33708194,-0.14092292,-1.1038417,-0.08961685,0.8091402,0.1112517,0.02656247,0.31560913,-0.1550345,0.18572994,-1.0769155,0.2721351,0.18981688,-0.5864409,-0.15321483,0.07063689,-0.18969655,-0.6880118,0.65666926,-0.20376055,-0.4630876,-0.13936332,-0.2939084,0.5339216,1.0099174,0.32066283,-0.5444067,0.16212149,0.15403162,-0.31769592,-0.24585359,0.51423454,0.114066,-0.39628226,-0.29411063,-1.1157777,-0.14635156,0.1463416,-0.21759894,-0.13711694,0.27957675,0.06745173,-0.11955167,0.35539603,0.07173479,-0.21622603,-0.46921423,0.07491127,-0.47726864,0.1958685,0.4618569,0.12828296,-0.34455845,-0.28381392,-0.18632439,-0.08693627,-0.02096391,-0.13164598,0.02042305,-0.15630075,-0.13438185,-0.23690097,-0.1744589,-0.3553839,-0.15600887,-0.08005294,-0.0544157,0.17518482,-0.18188433,-0.6745035,0.03794354,0.02264846,-0.26468414,-0.21049157,0.02443395,0.7858707,0.3085207,0.3426047,0.08580894,-0.83459675,-0.7321544,0.29390535,0.58377236,0.38627335,0.3583967,-0.47752854,0.13580689,-0.1390768,-0.0472422,-0.22234075,-0.6466335,-0.69349277,-0.60661834,0.24776804,0.04210765,-0.461634,0.18231136,0.45133832,0.1779425,-0.14479603,-0.13784878,-0.46758625,0.0158423,-0.3941278,0.16438846,0.27838227,-0.93074226,-0.12153659,-0.13759847,-0.1060136,0.6319782,0.5784075,0.41946194,0.2591278,-0.17781721,-0.25851884,-0.21999903,0.19757988,0.8273099,-0.41967708,0.386718,0.25575978,-0.05855953,-1.13395,0.67524546,-0.7458024,-1.0974418,-0.00183411,0.38509363,-0.03686714,-0.35352385,-0.9309869,-0.23078428,-0.06921761,-0.28396037,0.33314142]])
net.state_dict()['block2.0.bias'][:] = torch.tensor([-0.45695972,0.13282697,-0.09724394,0.3706998,0.027597265,-0.42888355,-0.18961087,-0.11994289,-0.69656587,-0.11414404,-0.28934464,-0.3644715,-0.2884142,-0.4523064,-0.39903286,0.083030015,-0.6429894,0.16011806,0.34512454,-0.6280401,-0.27081445,-0.019657847,-0.10037499,-0.3362698,-0.20094013,-0.07321697,0.1627557,-0.4665836,-0.29223332,-0.4529318,-0.3277536,-0.3568693,-0.19174956,0.06611946,-0.2198775,-0.51958525,0.2773933,-0.45936182,0.06634074,-0.06281985,-0.2025676,-0.7561613,-0.528674,-0.15554194,-0.5486091,-0.5265658,-0.83232754,-0.18956651,-0.14098275,-0.24266146,-0.2915999,-0.23343135,0.045369606,0.09928532,0.2884065,-0.23686637,-0.22202143,-0.163194,-0.60533184,-0.76275563,-0.3399168,-0.7922481,0.04799258,-0.21293898,0.2084421,-0.41172805,-0.18189041,0.5329846,-0.24538524,-0.39358336,-0.45765007,-0.012814365,0.19526665,0.2605219,-0.026269764,-0.510951,-0.6994306,0.16197653,0.14371577,-0.32490265,-0.3082134,0.024410019,-1.4274726,-0.30644888,0.07516025,-0.0030065505,0.045641374,0.21764089,0.21104309,-0.21179776,-0.40539324,-0.1472028,0.036081508,-0.18906808,-0.07108254,-0.47958195,0.33681738,-0.20061485,-0.6999942,-0.6747724,-0.12445736,0.357079,-0.013299409,-0.4699172,-0.5816515,-0.6055762,-0.5779329,-0.01210327,-0.520639,-1.0926381,0.96525455,0.7925984,-0.60567874,0.0384382,-0.4948044,-0.47339517,-0.35745105,-0.2611417,0.13686237,-0.21330802,0.46644711,-0.34946543,-0.32673267,-0.6184423,-0.47301137,-0.085407056,-0.4451764,-0.42774898])
net.state_dict()['block2.2.weight'][:] = torch.tensor([[3.39638352e-01,-2.01523349e-01,-3.10043991e-01,-2.09452227e-01,7.13562548e-01,6.38689026e-02,1.04790088e-05,4.87677962e-01,-1.82169840e-01,4.63455468e-01,2.03914233e-02,1.89933106e-01,-1.52321890e-01,-8.26153457e-02,2.53831685e-01,1.47264093e-01,3.04853439e-01,-5.00462390e-02,1.27026871e-01,9.23790395e-01,2.05663264e-01,-1.65264010e-01,-1.86242223e-01,4.89999115e-01,4.92258310e-01,-2.91141927e-01,2.65387803e-01,5.74500374e-02,-3.64851475e-01,3.59578043e-01,4.25263256e-01,1.44950822e-01,1.12735197e-01,-1.57803580e-01,-3.16981167e-01,4.48479317e-02,1.49233729e-01,8.74890760e-03,-5.94828844e-01,1.33199662e-01,4.37894076e-01,-7.05181658e-02,-5.54241776e-01,1.76206455e-01,1.76019356e-01,3.20544332e-01,1.75396100e-01,4.42732394e-01,1.85078636e-01,-3.18282805e-02,-1.81052789e-01,3.69024910e-02,5.26347756e-01,-1.45848304e-01,7.61469901e-02,2.78457642e-01,-2.59370446e-01,-1.60830975e-01,3.67687911e-01,-2.85898447e-01,4.50772613e-01,3.20267119e-02,1.30641028e-01,-7.37644266e-04,8.25761780e-02,-1.07459992e-01,2.85542101e-01,7.80649781e-01,3.22706729e-01,5.73900342e-01,5.54295778e-02,5.71751714e-01,-1.03165901e+00,4.41250533e-01,8.45536366e-02,4.57015365e-01,9.16596055e-02,2.24485472e-01,2.35134184e-01,5.55316508e-01,-8.66558775e-03,7.26816654e-01,-1.80399105e-01,1.92198232e-01,-7.42194951e-02,3.51858169e-01,1.07382704e-02,1.24769785e-01,1.82157978e-01,-5.44381551e-02,1.55004710e-01,4.40875649e-01,5.56503892e-01,-4.43819404e-01,4.04048741e-01,-1.31313773e-02,7.34342635e-01,-1.49289876e-01,3.65806818e-01,5.10717690e-01,-1.07006884e+00,-8.33838105e-01,5.46175063e-01,3.17477047e-01,-2.85596400e-01,-9.57549214e-02,1.13381080e-01,-7.49542490e-02,-4.88327518e-02,-1.30582362e-01,-6.90432563e-02,5.50561190e-01,-5.10880232e-01,1.68119356e-01,-1.77048475e-01,1.04833290e-01,1.68435618e-01,3.99455607e-01,2.14869663e-01,-4.54174161e-01,3.37212801e-01,4.59493876e-01,-1.44164905e-01,-2.40101591e-01,-8.16578977e-03,6.43425167e-01,1.47279292e-01,5.30728698e-01],[-0.11741272,0.4600899,0.11074987,-0.7427964,0.14252582,-0.64170086,-0.7124188,-0.15776166,0.44607317,-0.4432902,-0.48903573,-1.1795663,-0.0560318,-0.5866638,-0.2687018,-0.43113223,-0.63251555,-0.10400143,-0.27779585,0.00506138,-0.84418815,-0.9343779,-0.40185794,-0.6717614,-0.3542861,-0.8897598,0.22173414,0.06744441,-0.22653477,-0.16257073,0.0054562,-0.6614697,-0.46222347,0.21306694,-0.5197958,-0.09326776,-0.03550709,-0.08520088,-0.34528565,-0.44790524,0.20418094,-0.22391136,0.35622567,0.31522748,-0.4533739,-0.75418407,0.49496195,-0.2898023,-0.8466718,-0.36889207,-0.25847334,-1.0128096,-1.1808428,-0.84060246,0.1968506,-0.751499,-0.6558537,0.11067151,-0.29208043,0.06996324,-0.22337957,0.46358976,-0.42913294,0.09737876,-0.34018192,0.22349659,0.18037507,-0.6102232,-0.11616252,0.3808347,-0.5151812,-0.41930893,-0.93342113,0.24985033,-0.11910319,-0.6629173,-0.5018798,0.04754097,-0.0703719,0.45709836,0.1532215,-0.58867115,0.2231856,-0.10409461,0.3145215,0.05455098,0.04093024,-0.2258295,-0.1773067,0.3035209,0.40832418,-0.76923925,0.31666747,-0.13746661,-0.01513054,-0.36341473,-0.50544316,-0.18253179,-0.15483251,-0.6601446,-0.10562368,-0.02920747,-0.56023246,0.12409765,-0.35059667,-1.0282515,-0.3402387,-0.04225833,-0.9052671,-1.4669952,-0.36989248,-0.00465702,-0.52274066,-1.2506628,-0.48445827,-0.01342485,0.15223058,-0.16292877,0.02991566,0.0097131,0.6237781,-0.32277715,0.08587492,-0.11234341,-0.10460493,-0.61063933,-0.5223562,0.05648812],[-0.31848964,0.21713266,-0.04435984,-0.38446614,0.8820241,0.13023944,0.23905063,0.3243804,0.24279726,-0.27904752,-0.14671955,0.25840816,-0.23341013,0.41173384,-0.03246691,0.03065875,-0.13801718,0.6570595,-0.12947612,-0.46285337,-0.19045839,0.20791546,-0.49368924,-0.08439612,-0.2095397,-0.04395438,-0.00624749,-0.32404935,-0.2289713,0.14862764,0.24753949,-0.16687115,-0.7800937,-0.54903084,-0.24799089,-0.1321744,0.3284264,-0.2302993,0.42303166,-0.740473,0.19488852,-0.0923753,0.13776153,0.3077493,0.02655785,-0.03259106,-0.43027267,0.04173748,0.3917719,0.4448323,0.33533812,-0.06180451,-0.04848606,-0.07076522,-0.09402329,0.17190202,-0.8049304,0.4216203,0.88096154,-0.61370146,0.27862617,0.06523002,-0.06453269,-0.3781046,-0.05064481,-0.43613884,0.0830237,0.54209644,0.2095762,-0.22075234,-0.26191902,-0.43147433,0.05655048,-0.6442598,0.07007539,-0.41808084,-0.56772006,-0.15636681,-0.03516361,0.08957795,0.16280535,0.12123875,-0.03637457,0.19452941,0.36371475,0.00329993,-0.6581326,0.02605327,0.35211295,-0.09028193,-0.4588634,0.08226827,0.83075935,0.10306607,-0.7981808,0.33755514,0.07919129,0.10245077,-0.6746231,-0.28891566,0.11203173,0.01793094,0.15051386,-1.4198864,0.39893034,-0.08703424,0.59031993,-0.15204522,0.5199787,0.6185142,-0.22529477,-0.24643518,-0.01045299,0.0619054,-0.0300893,0.46577764,0.20336816,0.15754941,-1.0168327,0.21060096,-0.06012769,-0.15844728,-0.08727755,0.28595668,-0.2258229,0.17344348,-0.5350909,0.25103578],[-0.592616,-1.5679051,-0.33147192,-0.49232954,0.71440125,-0.1121217,-0.7300915,-0.6168513,-0.47297093,0.14674994,-0.1880481,0.20750716,-1.5780241,0.1997552,-0.76735806,0.50186145,0.333219,0.30896708,-0.284347,-0.49272832,-1.0088402,0.24953468,0.39432105,-0.19185662,0.24920566,0.21763597,-0.38053054,0.6307734,-0.5429001,-0.46518523,-0.98753667,-0.21771318,-0.22698551,0.3983712,-0.5148592,0.29862472,-0.47432607,-1.6895076,-0.02190702,-0.5074149,0.3942706,-1.0627387,-0.35657144,0.2152666,-0.48124436,-1.207763,0.3441565,-1.0303158,-0.29870704,-0.80851525,-0.6759233,-0.37874115,0.71538943,-0.17292076,0.34497336,0.21162865,0.25504315,-0.11042475,-0.20167574,-0.06646434,-0.65145594,-1.64416,-0.32547486,-0.85023654,0.2598137,0.06255747,-0.11473986,-0.02769219,-0.01010671,0.10206082,-2.287001,-0.68333316,0.03376865,0.06449109,-0.24173589,-0.9909428,-0.5894483,0.09253436,-0.2024351,0.16872297,0.7701887,0.1402629,-0.933971,0.3377786,-0.66076314,-0.31916025,-0.44714215,0.1244211,-0.87316465,-0.24661885,-0.1059053,0.4102088,-0.0761976,-0.01538747,-0.20108667,0.07195578,-0.36927232,-0.6163761,-0.24179924,-0.729843,-0.65416366,-0.6278919,-0.25153905,-0.18475614,0.7926971,-0.4028054,-0.15590693,-0.17717597,-1.2854073,0.24578093,-2.4383638,0.3679591,-0.7816411,-0.18938002,-1.7506596,0.00843304,-2.2186687,0.22680119,0.26678783,0.07213673,0.21883832,-0.8663349,-0.13580114,-2.9671693,-0.9775769,0.83003104,-0.9264089,0.19652337],[0.35778144,0.08787153,-0.01963369,-0.13699456,-0.41664502,-0.26612118,-0.02992137,-0.5699501,-0.13280317,-0.16314767,-0.22295922,-0.7230386,-0.46519715,-0.02973412,-0.171403,-0.06940135,-0.33722508,-1.1002191,-0.9924662,-0.7439649,0.34191582,-0.2949851,-1.3808084,-0.08591104,-0.8075232,-0.52537477,-0.80645764,-1.0652138,-0.33225253,-0.06731755,-0.3215961,0.17467031,-0.35966063,-0.6976073,0.22131355,-0.76920795,-0.6353458,-0.57607555,-0.42998165,-0.22800416,-0.52672774,0.00203179,-0.3964332,-0.5813268,0.3962339,-0.41706985,-0.7660384,-0.65251535,0.2834117,-0.20682718,1.0234554,-0.71307343,-0.601155,0.41886774,-0.6242922,0.1810052,-0.41947806,0.30935788,-0.28869778,-0.02743307,-0.29930136,-0.7384927,0.38631514,0.05977001,-0.39671502,-0.34987077,-0.5237061,-0.37011424,-0.30621868,-0.80598605,-0.22369352,-0.11119968,-0.43837366,-1.0737911,-0.14924686,0.6278699,0.15804076,-0.2785332,0.05957505,-0.68991303,-1.1047231,-1.1210631,-0.284553,-1.1653944,0.02747731,0.13181466,-0.44630826,0.42965055,-0.89423364,-0.4414366,-0.47167218,-0.39077836,-0.53335905,-0.17810008,0.13876536,-0.7795259,-0.3892591,-0.3681413,-0.20495541,-0.48114175,-0.31303263,-0.47474605,-1.1546421,0.2672748,0.5429905,-0.07727539,-0.71580917,-1.0393109,-0.16354871,-0.25540906,-0.45396385,0.41648388,-0.23885725,-0.56688946,-0.20508114,0.2238399,0.21592538,-0.25975266,-0.2788262,-0.52946436,-0.7564436,-0.06656206,-0.4291243,0.02666179,0.17866112,-0.6642963,-0.22601338,-1.0193875],[-0.81174886,-0.22735909,0.7923148,-0.184705,-0.54769677,-0.835401,0.3245059,-0.74917287,0.00385197,-0.07507712,-0.3868176,-0.33369535,-0.36813506,0.72413903,-0.3313099,-0.7086421,-0.5832789,0.09047923,-0.32305023,-0.7193782,0.11302144,-0.50893015,0.98960364,0.21250363,0.1742079,0.42802396,-0.07484815,-0.6701964,-0.05804819,-0.5339285,-0.22092094,-0.4786448,-0.06484064,-0.86204547,0.33324808,-0.01270212,-0.90159243,-0.9486039,0.02192387,0.16794838,0.41290703,0.01369677,0.60464597,-0.36640638,0.1311173,-0.40251493,0.32130978,-0.64618343,0.07026876,-0.38252947,-0.46322167,0.63402224,-0.5449429,0.7158985,-0.3347658,-0.02066403,-0.4970994,0.17678142,-0.49892294,-0.47091314,-0.8372709,0.14281473,-0.9392734,-0.12602483,-0.10581446,-0.05414674,-0.5769499,-0.42168376,-0.24442375,-0.26056644,-0.46134645,0.19010623,0.03364453,-0.40913078,0.08793096,-0.36293593,-0.03323703,-0.459977,-0.30244404,-0.29739922,0.02652782,0.03824997,0.1788906,-0.02609809,0.02383564,-0.10158242,0.06092416,-0.5590598,-0.00258178,-0.15211308,-0.55611056,-0.07348771,0.1159485,0.23499326,0.17258444,0.23156863,-0.2655902,-0.34861612,-0.47255558,0.23396859,-0.25557342,0.6187272,-0.11073108,-1.0189028,0.27983674,0.01525964,0.22320727,0.73043025,0.00166765,-0.05202469,-0.46544906,-0.05970845,-0.01306252,0.03836084,0.04028969,0.27232063,-0.5547457,0.05284872,-0.18375847,-0.03300849,-0.26114464,-0.29504868,0.8556983,0.06895234,-0.16113797,-0.10388984,-0.1645547,-0.3077739],[-0.2540418,0.33242756,-0.05351948,-0.23365216,-0.13402686,-0.14800008,-0.04884835,0.37419975,0.07910564,0.370374,-0.8231137,-0.12363022,-0.2500114,0.2705567,-0.47100574,0.19752829,0.75352556,0.34434745,-0.06347308,-0.09346079,-1.2725595,-0.32703358,-0.5918437,-0.0292898,-0.21454877,0.12291799,0.40206787,0.02274247,-0.5356432,-0.4977946,0.26024482,0.22847216,-0.0280973,0.12425081,-0.01660092,0.38916948,0.53502756,-0.9497095,0.07731996,-0.4647145,0.06379828,-0.56509626,0.0715645,-0.4989483,0.4455003,-0.23355526,-0.8145188,0.6199147,-0.33560437,-0.6151712,-0.1691201,0.45353475,0.1043146,0.08543544,-0.29085156,-0.37032434,-0.759164,0.06994534,-0.33077928,0.09635263,0.1490393,-0.15555139,-0.04536097,0.03144697,-0.09274267,-0.1311044,-0.33141676,0.21258548,-0.31138054,0.08651407,-0.4967611,-0.5872416,-0.1340786,-0.84755284,-1.0082654,-0.2649872,0.20579556,-0.24223588,0.10714278,0.30076438,0.04168467,-0.64697874,0.3542138,-0.8339426,-0.3883142,-0.16086523,-0.3100141,0.1157698,-0.8356305,-0.02136784,0.05852516,-0.10244865,-0.01929422,-0.2752059,0.12471532,0.3066377,0.07031265,0.11584935,0.12384092,-0.43386972,-0.22680286,-0.11680211,-0.2435436,-0.02447769,0.3883453,-0.562395,0.22761506,-1.2405915,-0.9441349,0.04107141,-0.47826776,-0.31466758,-0.16445222,0.13889158,-0.61713815,-0.35211992,-0.00768243,0.34507826,-0.12715375,-0.5710138,-0.02585224,0.05694484,0.05751362,0.3644037,-0.54306805,-0.00766677,-0.5266148,-0.07255081],[-3.19980323e-01,3.47092450e-01,-6.10881269e-01,-1.15227170e-01,-1.43271863e-01,-1.59581229e-01,-4.87698317e-01,-2.35601753e-01,-9.80093062e-01,2.86953419e-01,-1.54907894e+00,-8.03296626e-01,-4.96036351e-01,5.54515839e-01,5.79758048e-01,-1.51247299e+00,-4.26822782e-01,-1.18450187e-01,-4.68189389e-01,4.63859260e-01,-1.76469475e-01,1.14989072e-01,-4.07931358e-01,3.41993660e-01,-6.16130494e-02,-6.71081245e-02,5.08768737e-01,4.34734151e-02,1.71528444e-01,-8.34293783e-01,-8.50913767e-03,-1.36801198e-01,1.01854444e-01,-5.44560671e-01,-1.16259706e+00,-9.30894494e-01,-4.77982879e-01,1.00477576e-01,-4.14476424e-01,-3.36339206e-01,-8.46306860e-01,5.61111867e-01,-1.22125161e+00,1.51553124e-01,-3.50110531e-01,-4.97834653e-01,-8.24891090e-01,4.70918491e-02,-9.41549957e-01,-2.72900522e-01,-5.88778675e-01,-6.51302189e-03,5.30169845e-01,1.69726685e-01,-1.24051492e-03,-2.55009502e-01,1.08299449e-01,-3.61768395e-01,-5.48065484e-01,-6.06250525e-01,-1.30902618e-01,1.45449758e-01,-2.24557802e-01,-4.62987751e-01,-7.51586258e-01,-1.24045479e+00,-1.60797966e+00,-7.07279325e-01,1.01724016e-02,-1.73252976e+00,-1.18978524e+00,-5.33933677e-02,-2.50945717e-01,-8.42598021e-01,-1.46612823e-01,-8.61697555e-01,4.53257933e-02,1.48893520e-01,4.14522320e-01,-5.97455688e-02,-2.14416552e-02,-5.50684392e-01,-2.14297622e-01,-7.37812072e-02,-6.34620905e-01,3.02893460e-01,2.86299944e-01,2.84873784e-01,-8.53219450e-01,-2.86020964e-01,2.55010545e-01,-2.75839776e-01,3.69264811e-01,-4.57248360e-01,-2.00835317e-01,-5.92186570e-01,7.79346749e-02,-1.42482352e+00,-1.33759832e+00,2.53706992e-01,-3.90118569e-01,3.10615897e-01,-2.93268949e-01,9.10039172e-02,3.81865829e-01,-2.81757563e-01,-8.81199837e-01,-2.74814904e-01,6.49249628e-02,2.78089494e-02,-3.86770666e-01,-1.33378968e-01,5.13285160e-01,3.24813753e-01,-1.84818998e-01,6.74508095e-01,-2.14196533e-01,-4.24573004e-01,-1.31778300e+00,-1.56942829e-01,5.30890465e-01,1.15105772e+00,-8.07405233e-01,-5.23141511e-02,-1.02013850e+00,9.39839408e-02,3.08404386e-01,-2.48514831e-01],[0.07177297,-0.3472603,0.13424386,-0.6361177,0.09725611,0.68638986,0.48712143,-0.00258234,0.11340337,0.43657365,0.02228346,-0.19322795,-0.3399191,-0.21572217,-1.3318034,-0.4713807,-0.01772615,-0.26186532,-0.33761042,-0.5570389,-0.35825554,-0.26289538,-0.20429876,0.14678574,-0.1841934,-0.12651886,-0.8844417,0.26734143,0.16275436,-0.09322324,-0.17908505,-1.2206496,-0.6182573,0.47437254,-0.6272887,-0.8919982,-0.8703935,0.2768899,-0.4505343,-1.1079453,-1.0849959,0.30733916,-0.45024645,-0.5251783,0.17183495,-1.0729036,-0.47254923,-0.02584716,-0.26570728,-0.7321676,-0.6830836,-0.32023746,0.86203027,-0.51169866,0.35510173,-0.72194034,-1.0677134,-0.45654613,-0.343158,-0.3757371,-0.13279343,-0.8078872,-0.04354989,-0.3896414,-0.5113582,0.24660349,-0.72377783,-0.48944756,-0.15673012,-0.780831,-0.05909834,-0.46018073,0.14118406,-0.15879868,-0.6445078,-0.72235453,-0.5005784,-0.01850077,-0.8227363,-0.08985095,-0.54300416,-0.7914284,-0.2437141,-0.1351435,-0.5446377,0.13721982,-0.64975554,0.7974929,0.3437786,-0.63159716,-0.09695013,-0.37148508,-0.04234368,-0.6416311,0.75378007,-0.81860065,-0.04981611,0.2848089,0.17450315,0.53083664,-0.87256515,-0.4630085,-0.7757132,0.03728426,-1.0438623,0.01386994,-0.405891,-0.40197614,0.38934305,-0.15793322,-0.42456865,-0.01891572,0.16312546,0.09392442,-1.3094323,-0.30222544,-0.5431186,-0.4354577,-0.2801174,-0.16422844,-0.33772495,-0.23827592,-0.390733,-1.1551949,0.8512147,0.36892998,0.2916389,-1.1782253],[-3.2955740e-02,-7.0662111e-01,-1.0048905e-01,-2.9390693e-01,-1.5271778e-01,-6.7157632e-01,3.9996970e-01,1.2508725e-01,-2.4706487e-01,-4.2368513e-01,-6.6830033e-01,-3.5280979e-01,3.7728116e-01,-8.4344629e-04,-6.2842166e-01,-5.1930404e-01,-3.4449184e-01,-3.1231594e-01,-2.8223362e-02,-2.6375678e-01,-5.1342005e-01,-5.4526234e-01,5.1037166e-02,-7.6199210e-01,-5.0757009e-01,2.5664151e-02,8.6068851e-01,-6.3519847e-01,6.9009918e-01,-5.3761321e-01,-2.3939961e-01,-9.7853452e-01,1.3799214e-01,-1.2250414e+00,-4.7789216e-01,-6.2634993e-01,-8.5290670e-01,-3.3799138e-02,-4.0573752e-01,-2.4500082e-01,-5.6786376e-01,6.3520158e-03,2.4211393e-01,-4.5775300e-01,-2.1146083e-02,-1.5566458e-01,-6.0011899e-01,-5.1527292e-01,-4.0209806e-01,-4.7807333e-01,2.6433983e-01,-3.1825429e-01,-3.7129704e-02,-5.7960594e-01,-7.8611553e-02,-7.0674247e-01,-7.6905364e-01,-1.3116115e-01,-5.5700801e-02,3.1190351e-02,-3.4948513e-01,2.9400101e-01,-4.3880859e-01,6.4381406e-02,-2.1398199e-01,-2.7979684e-01,-4.2437503e-01,1.8355145e-01,-2.4372466e-01,-5.0580060e-01,-1.6327405e-01,2.2721066e-01,-9.8799372e-01,-7.9604101e-01,5.3447258e-01,-4.0708229e-01,1.9436730e-01,-2.8580146e+00,-8.8301426e-01,-5.8156246e-01,-1.1228511e+00,-4.5725936e-01,-1.4659594e-01,-2.0399139e+00,9.4672376e-01,4.4141194e-01,5.4586668e-02,-3.2601446e-01,-2.9533508e-01,-6.1474162e-01,-1.3076909e-01,-1.8610071e-01,-4.8773885e-01,-2.4616270e-01,-8.5492782e-02,-2.3142318e-01,-3.7659463e-01,-1.8973599e-01,-7.8238308e-01,4.7380140e-01,-5.1701111e-01,-2.0425633e-01,7.4035272e-02,-2.0836391e-01,-8.2970315e-01,-9.6796751e-02,-6.6440350e-01,-1.0816168e+00,2.9313242e-01,-1.8212835e-01,-3.7775660e-01,-1.5714583e-01,-2.5751305e-01,-2.9590774e-02,3.5339868e-01,-7.0175564e-01,-4.7620657e-01,-3.3995235e-01,3.8246125e-01,-6.2698793e-01,-1.0115530e+00,-6.4667201e-01,7.8690752e-02,-3.4752396e-01,-1.8542236e-01,4.2901266e-02,7.2178997e-02,-3.2987723e-01],[0.07580923,-0.26674062,-0.27531043,0.49103534,0.4160593,0.0468929,-0.05834339,0.23955634,-0.32028246,0.2490466,0.12578699,0.46499798,-0.0322662,0.4136582,-0.27250206,0.3806824,0.34906483,0.11149623,0.18099442,-0.06849489,-0.28411344,0.13870683,0.07122834,0.64320606,-0.11538071,-0.29735082,0.47947165,-0.1399487,0.24095775,0.14643376,0.7025543,0.5134077,0.3542482,-0.16417119,0.46328926,0.20840694,-0.26260614,0.00215289,0.429989,0.2395068,-0.09228284,0.14371543,0.40597263,-0.2246526,0.5266506,-0.00439699,-0.16474344,0.04854661,0.39132905,-0.55529934,0.29884034,0.06268895,0.10954238,0.21851005,0.21430995,0.00824998,-0.22884618,-0.05305287,-0.10117461,-0.5240604,-0.52976376,-0.3153706,-0.01186155,-0.52711403,-0.40071592,0.15365452,-0.21357231,0.47314024,-0.4066842,-0.58400023,-0.11961465,0.37415454,0.05974591,0.02726689,-0.04230854,-0.01238982,-0.5883235,0.19174848,0.5083052,0.04590457,0.18188111,-0.189585,0.25035238,0.13234706,-0.18343101,0.21240863,-0.56831187,0.26295894,-0.44373572,0.3718723,0.12970954,0.30970913,0.03156519,0.34188533,0.2000197,-0.59366214,0.22060156,0.2035988,0.11755477,-0.22344258,0.37841645,0.3556286,0.68069154,-0.2038616,-0.47006428,-0.16068017,0.06424198,0.24271213,-0.07335297,0.39572102,0.12854607,0.59474826,-0.1245941,0.46770358,-0.0848327,-0.02234036,0.13626504,0.13494167,0.7315063,-0.03149972,0.08816583,0.17363429,-0.2993104,-0.31021827,-0.30138186,0.3276221,0.18798696,-0.12857103],[-4.22506362e-01,2.23038405e-01,4.56871718e-01,6.13915967e-03,1.82756096e-01,-5.08506536e-01,1.74258977e-01,5.59440792e-01,6.37777388e-01,-4.43779677e-01,6.21288605e-02,-5.85064411e-01,-1.63154155e-01,3.91455412e-01,1.36875018e-01,4.58006799e-01,3.48135196e-02,1.95814855e-02,-3.41220170e-01,3.86500329e-01,-6.87824368e-01,-1.33707747e-01,-3.84779386e-02,4.01926249e-01,-6.10031039e-02,-3.98279466e-02,-1.70869976e-01,7.07846601e-04,6.59121796e-02,-1.05905247e+00,3.65641356e-01,1.41569093e-01,3.11192244e-01,2.27843702e-01,-4.12622869e-01,-1.46259516e-01,-1.44698441e-01,-3.16185392e-02,-1.88397974e-01,-3.65921497e-01,-3.64419855e-02,1.10773571e-01,1.57749042e-01,4.99822438e-01,-1.05051860e-01,-3.73112172e-01,1.68626845e-01,1.61508217e-01,-4.00855124e-01,-7.02561811e-02,2.51384646e-01,-2.98220038e-01,-5.02134502e-01,-1.27649799e-01,-7.52564013e-01,-3.30160975e-01,4.68015194e-01,1.96481794e-01,-3.88149172e-02,-5.80708444e-01,1.26502886e-01,-1.65212870e-01,-7.79716596e-02,-1.22697987e-01,-2.86760241e-01,1.93501443e-01,4.18670615e-03,2.93524802e-01,2.46544242e-01,-1.07126147e-01,1.14403963e-01,2.04802930e-01,-6.52497634e-02,6.22758381e-02,5.94796650e-02,-2.53987581e-01,-3.97575378e-01,-4.98434961e-01,-2.62210190e-01,-1.18722320e-01,8.89526494e-03,5.36843054e-02,-4.06716540e-02,-2.36888662e-01,6.61547109e-02,1.44244939e-01,-1.29244298e-01,-2.24083543e-01,-3.60687114e-02,-8.66628066e-03,-5.95118701e-01,-1.97720051e-01,-1.20504640e-01,-1.89123929e-01,-3.10698807e-01,4.14306559e-02,9.89772193e-03,1.82902038e-01,-2.29596421e-01,3.38792540e-02,-4.21913266e-01,-1.12912565e-01,2.41219819e-01,-1.46034122e-01,-3.53393584e-01,-1.64976910e-01,1.75232887e-01,-2.08278596e-02,-8.24053586e-02,-2.35589724e-02,2.62564067e-02,-4.01922345e-01,-1.78051785e-01,1.17901608e-01,2.29222357e-01,-1.40225112e-01,1.20265558e-01,1.00744240e-01,-2.06382751e-01,4.23172116e-02,-3.69647563e-01,-1.41550764e-01,-3.03107798e-01,8.25685039e-02,1.73146501e-01,8.22796486e-03,1.42240003e-01,6.22904539e-01],[-1.4984537,-0.12462682,-0.36585057,-0.00167512,-0.06646661,0.3068071,-0.03482436,0.35005265,-0.5805402,0.432352,-0.23204406,0.2085755,0.2500752,-0.13892841,0.91893685,0.11978704,0.0024478,-0.36176038,-0.7689052,-0.13826166,0.33568963,0.20099929,0.19196142,-0.15524478,0.04980309,0.5875172,0.39080235,-0.1326292,-0.87642395,0.08538003,-0.57972515,0.4383642,0.06717001,-0.6955057,-0.11033723,-0.25974643,-0.09616034,0.32263345,0.46867567,0.0501081,0.41036794,0.06254517,0.09077713,-0.0052228,0.22931129,-0.21753423,-1.0542598,-0.42364895,0.40419015,-0.1232355,-0.42311144,0.0736518,0.28507575,-0.44263166,-0.03317664,0.36029938,-0.11860977,-0.89225715,-0.54344267,0.08170265,-0.5553253,-0.03356842,-0.5853851,-0.50577664,-0.10328951,-0.38048106,-0.8196337,-0.45980704,-0.15546997,-0.32077616,-0.50732803,-0.8717295,-0.6298477,0.19614255,0.07453701,-0.35054666,0.54624724,0.0499352,-0.31135142,0.24078266,-0.1706103,-0.01197407,0.05504686,0.5663269,-0.20974034,0.19867267,-0.02720592,0.48361433,-0.9800415,0.21155965,-0.3166689,-0.5431695,0.40713757,-0.72077554,-0.8813525,0.01455728,-0.34039602,-0.9444831,0.12766063,-0.43346593,0.06728524,0.01519686,-0.05800169,-0.23041481,0.53597313,0.47811896,-0.37554568,-0.11046382,0.34774986,0.04061536,-0.25603643,-0.20692477,0.4305934,-0.04977152,-0.39970067,0.3848655,-0.14190054,-0.59686005,-0.65496796,-0.27721432,-0.11366347,0.08954043,0.12357602,0.11911679,0.26528275,-0.573217,-0.02609042,0.10653199],[-1.1454699,-1.5124885,-0.44771272,-1.0917298,-0.24753864,-0.7471013,-0.5589131,0.2073376,-0.08347038,-0.6155368,0.67273456,-0.2519959,0.27227,-0.2548917,-0.13712947,0.3984332,-0.01080842,0.39001307,0.286631,-0.34084004,-0.09675629,-0.7044712,-0.6520761,0.34151387,-0.11348673,-0.09196478,0.04694619,-0.48683715,-0.13679528,-0.47492054,-1.0812787,-0.7345732,0.14724784,0.25230646,-0.3258863,-0.428537,-1.10697,-0.4111826,-0.14377859,-0.01770882,0.01250784,-0.7312054,-0.40731475,-0.59306824,-0.60067993,-0.58513623,0.35671172,-1.2680001,0.11191529,-0.47183964,-0.5402227,-0.8018948,-0.30289918,-0.21265562,-0.82251525,-0.4021469,-1.0091902,-0.76537246,-0.06824589,-0.5046316,-0.0740947,-1.1744677,0.34592053,-0.29733348,0.49448836,-0.10865426,0.3761165,-0.2948953,0.16733879,-0.4337822,-0.22625574,-0.27502888,-0.5385225,0.72418404,0.36925668,-0.16911963,0.02868116,0.00887246,-1.0368928,-0.46950307,-0.22961345,0.35774413,-0.18039104,0.11920097,-0.96503294,0.20767643,-0.69193053,-0.14270717,0.54978335,-1.1169378,0.07084972,-0.18279716,-0.6681739,-0.35343432,-0.22580077,-0.3452977,-0.3924625,-0.17967428,-0.45287788,0.35947892,-0.02154209,-0.3963063,-0.67700607,0.22319669,1.7415504,0.05084518,-0.19332306,-0.8496929,-0.2643708,0.06070415,-0.235131,-0.24384525,-0.01779857,0.25569853,-0.03584045,0.23230453,-0.82226187,-0.13198888,-0.00854935,-0.6658077,-0.3782038,-0.41086787,-0.8567614,0.00429399,-0.35604474,0.03306374,-0.4721871,0.02805719],[0.2506443,-0.31400973,0.07323223,-0.81018716,-0.8875236,-1.6245635,-1.6923271,0.10388836,0.04252157,-0.8880553,-0.06237,0.33617637,0.59348506,0.07306547,0.52349555,0.66923296,0.2136059,-0.45223287,0.7623677,0.03105627,0.079555,-0.9255808,-0.778484,-1.2209847,-1.5130064,-0.7077857,-0.5737881,-1.0773994,0.59652144,-0.37196356,-0.57973254,0.33138236,0.48176482,-0.14759177,-0.37613854,-1.0699017,-0.42369795,-1.1960788,-0.63753915,-0.39252916,-0.54817563,-1.3349153,-0.11137936,0.05977061,-0.8909698,-0.00512331,0.12350106,0.45651534,-0.159401,-0.1109519,0.92888486,0.5832413,-0.07776025,-0.16013578,-0.19488128,-0.59189266,-0.16585803,-0.58697367,-0.22449422,-0.86601293,-0.6595918,-0.5723037,-0.19146343,-0.5714201,0.37283584,0.10522189,-0.42943186,-0.7753261,-0.7714447,0.7184437,-0.77018106,-0.32691553,-1.0775836,0.00943528,0.33916432,-0.9514095,-1.4355967,-0.0956776,-0.3269488,-0.12005153,0.5323415,-0.47860226,-0.02818111,0.04240557,1.0412933,0.20233226,-0.8236167,-0.30011073,-1.2231733,-1.1697471,-0.8248758,-0.8112308,-1.2705805,-1.0892212,0.6062813,-0.35961986,-0.2111217,0.05884722,-0.44926906,-0.4373179,-0.6104656,0.07028189,-0.41626933,-0.27259174,-0.80927795,-0.6189872,0.69309336,-0.8108947,0.16221958,-0.09160386,-0.5131013,-0.25523236,-0.2871736,0.4289645,-0.62331265,0.38390362,-1.5595859,-0.5262974,0.09902199,-0.7273292,-0.45219696,-0.70782924,-0.87465423,-0.11134206,-0.40980372,0.2767534,0.01620866,-0.6425595],[-0.23295885,0.07370231,-0.4279783,-0.30121443,-0.08415796,-0.05542805,0.13205042,0.252896,0.03842228,-0.20260032,0.2833746,-0.28451163,0.4090681,-0.16004583,0.07530948,-0.4275841,0.14583485,0.31403413,0.07095676,-0.14007697,0.50716394,0.19803236,-0.9919433,0.40030992,-0.41286457,-0.29601097,0.22626156,-0.01016461,-0.19789217,-0.60380024,0.03434857,-0.22740343,-0.2084519,-0.47156304,0.12600505,0.3229122,-0.05547135,-0.22929063,-0.28157356,0.17194284,0.22163723,0.49374172,-0.86468405,0.08811114,-0.50650907,-0.2889233,0.27650484,0.11027554,-0.03560545,0.97313,0.34758857,0.18508813,0.19058082,-0.0083145,-0.1754096,-0.65179145,-0.0952667,0.19406301,-0.29358786,0.61375207,-0.37799206,-0.19909382,-0.7865904,-0.62052464,0.20233518,-0.46369302,0.13786387,0.16687548,0.04736863,0.21225618,0.48014095,-0.42464098,-1.4673431,0.23671637,0.01287645,0.21855886,0.7281364,-0.16126716,0.31310424,-0.52325165,-0.35116366,-0.42433435,0.54594535,0.07416291,-0.39594585,-0.01450522,-0.38358498,0.00639211,0.26775968,-0.72105443,0.04174064,-0.19119291,0.10510947,-0.6257363,-0.07975287,0.33865097,0.06850825,-0.6598132,0.18390459,-0.12441436,-0.58823997,-0.15844207,0.13460284,0.28893048,-0.2632944,-0.26510772,-0.21383111,0.17454906,0.16680977,-0.4256862,-0.30966043,-0.09558804,-0.36537093,-0.67994654,0.10937163,-0.00226373,0.15153514,0.24937409,0.38150468,-0.6524587,0.18622997,0.24584705,0.34096903,0.30678898,0.04216773,-0.2605671,-0.5452232,-0.470354],[0.00231722,0.4834486,0.5311274,-0.3089885,0.2503182,-0.70301795,-0.140547,-0.00136872,0.17975946,-0.09065315,0.04405335,-0.45123106,-0.1272255,-0.06128743,0.21296418,0.51924556,0.04731403,0.4091066,0.08307905,0.02292266,0.21106958,0.01906518,-0.13719922,0.03335247,-0.09363791,0.25885427,0.03411757,-0.03491001,0.0640924,0.21503271,-0.32704273,-0.08119965,0.45546433,0.11873038,0.25494382,-0.0837311,0.3480117,-0.09467287,0.2086591,0.16075912,0.01083647,-0.03903396,0.09070154,0.5132463,0.3093596,-0.00456625,-0.553527,-0.24769376,0.52961373,-0.82579225,-0.5089629,-0.20967823,0.46064693,-0.3036842,0.08025046,-0.1883966,0.393874,-0.16204382,-0.06163121,-0.34716603,0.19299741,-0.1430779,0.25406146,0.07117341,-0.3614115,0.20287375,0.6644831,1.0515848,0.12922953,-0.18548988,-0.19623776,0.26546595,-0.2643306,0.05942364,-0.1230185,-0.1427547,0.94477665,-0.3485093,-0.21653655,-0.583111,0.21171837,0.18339989,-0.16407678,-0.09802988,-0.53432405,-0.4139731,-0.23362845,-0.1508057,-0.10617504,0.23040764,0.33392054,0.5316166,0.10742609,-0.01777911,0.06556199,0.08360214,-0.33723894,0.06610619,0.23997349,0.14953278,-0.6513299,0.00155933,0.00963539,0.18316422,0.34105116,-0.60719204,0.41933954,0.16991927,0.17192335,0.92109936,0.08658702,-0.4064385,0.06651658,-0.00688814,-0.07434224,-0.37361032,-0.16667895,-0.3026796,0.45449913,0.28618506,-0.08544593,0.47845823,-0.44250724,-0.28117666,-0.01106097,-0.3443978,-0.5739763,0.17948069],[0.02649101,-0.40652964,-0.7007411,-0.20426309,-0.47671932,0.1945273,-0.48743293,-0.11354933,0.00569426,-1.0789593,0.11212492,0.6242184,-0.21302077,-0.17428757,0.15255733,-0.48813418,-0.480717,-0.29274598,0.13885158,-0.00172972,-0.0767828,-0.47400928,0.12982336,-0.37772867,-0.1644009,0.08425164,-0.42907277,-0.16188696,-0.27781498,-0.540143,-0.89169765,-0.13533045,-0.669244,-0.13671151,0.15320079,-0.18390287,-0.7009914,0.19124109,-0.10282632,0.3061667,0.02474672,0.02320554,0.01791555,-0.68932515,-0.0705994,-0.10178264,-0.12227817,0.11670457,-0.8678621,-0.26343626,-0.1033222,-0.3341499,-0.4810776,-0.16826211,-0.20424235,0.34291437,0.10392782,-0.26562077,0.15946957,-0.61182386,-0.32366532,-0.02167682,-0.26867673,-0.383873,0.03398694,-0.07799059,0.21726637,-0.2704333,-0.35409027,-0.3292166,-0.7382718,-0.14428388,-0.67873496,-0.4946621,-1.1577445,-1.2253325,-0.48662564,0.09109165,-0.541848,0.24024655,0.00606183,-0.03684301,-0.5664328,-0.25721943,-0.5852301,0.12703073,-0.99706006,0.25887445,-0.5523906,0.3098738,-0.25972298,1.1564318,-0.55509806,-0.4627752,-0.5836655,-0.20070471,-0.40007174,-0.16885936,0.13049167,0.00250221,-0.17927623,-0.68719435,-0.36504838,-0.25851968,-0.19582593,-0.4636633,-0.35329732,-0.21983047,-0.6050431,0.18265207,-0.46134117,-0.387652,-0.66057897,0.00464167,0.11360319,-0.43889782,0.02452341,-0.19203155,-0.0532024,-0.49345487,-1.2632289,-0.34423012,0.02322143,-0.11912642,-0.12005809,-0.43479234,0.01120523,-0.6932459],[0.12572934,0.25164178,-0.5386647,0.06604565,-0.31038055,-0.04418508,-0.2547621,-0.21784955,-0.02558071,0.09094284,-0.6887308,-0.2738832,0.23997872,0.25532645,-0.24803948,0.41617155,-0.7428543,-0.6335293,0.26657325,0.06022502,-1.0609015,-0.5104846,-0.1884035,-0.19606733,-0.21974833,-0.28907734,0.5096711,-0.25340024,-0.7674041,0.2530916,-0.25257108,-0.19457455,0.6326414,-0.2585794,0.08134849,-0.33360776,0.20483336,0.1911445,-0.7476511,-0.12483964,0.19138144,-0.14175726,-0.6404922,0.44397122,0.14473939,0.35623345,-0.33469954,-0.18098995,-0.65123266,-0.12878056,-0.2824135,-0.06700318,-0.23979945,0.03499179,0.2509266,-0.4383209,0.31688714,0.4929,0.01153237,-0.78185695,-0.04749476,-0.87390864,0.38002512,-0.33382162,0.39311165,0.04460376,-0.13900504,-1.0858046,0.10579612,0.20222728,-0.14791346,-1.2375861,-0.98114884,0.26801953,0.41921088,-0.21842954,0.56872475,-0.22387555,0.3671087,0.43001884,-0.50871855,-0.18745495,0.65126085,-0.10570901,-0.2599213,0.59599465,-0.24705343,-0.57486576,-0.28100547,-0.89157456,-0.05613976,-0.23183863,-0.49667662,0.0533582,0.22902066,-0.5824578,0.4658674,0.38253087,-0.5312613,0.12678096,-0.30109066,-0.15617086,-0.03193666,0.21179497,0.04333832,-0.1350001,-0.09148386,-0.2676526,0.46095723,0.27692303,-0.5301941,0.08566846,0.25384247,-0.50305045,-0.3248624,0.1616999,-0.68158,0.47379068,-0.17364122,-0.52889425,-0.01205737,0.9091015,0.07468866,0.0721717,-0.05651785,-0.24484737,-0.02067821,-0.14419444],[-7.66016424e-01,-7.96963349e-02,-5.01906037e-01,-1.83476374e-01,-6.10354364e-01,-4.88004416e-01,-4.84139949e-01,7.12407604e-02,-1.19476743e-01,-3.09787512e-01,-4.20342654e-01,-7.16540337e-01,-7.48346210e-01,-2.91754425e-01,-2.27150410e-01,1.76381037e-01,-1.87605813e-01,-3.09330106e-01,-1.71104312e+00,1.41950965e-01,-8.62071574e-01,-8.96092176e-01,-7.36617893e-02,-9.63046178e-02,-7.28865325e-01,-9.43820596e-01,-5.45902312e-01,-5.32060742e-01,-1.46965548e-01,-1.71318427e-01,6.56987786e-01,-7.72100687e-02,-1.78998396e-01,-6.54826164e-01,-2.83086509e-01,-1.25424135e+00,-4.23510820e-01,-1.22287925e-02,-7.63489068e-01,-5.68552554e-01,-6.02210224e-01,-2.20020097e-02,-7.49789774e-01,-4.72502232e-01,-5.34999669e-01,3.00663766e-02,4.87555414e-01,-1.11085880e+00,-5.93982451e-02,-1.96914673e-01,7.20489472e-02,8.64725858e-02,-1.20547712e+00,-3.93707126e-01,-1.02115107e+00,2.02037513e-01,4.56075594e-02,1.54273491e-02,-6.59842730e-01,-6.55417979e-01,-7.77627081e-02,2.26892442e-01,-1.85210973e-01,-3.18220913e-01,-6.57342523e-02,-2.65081730e-02,-8.47762465e-01,-4.23332691e-01,-5.52185953e-01,-2.99858868e-01,1.31228924e-01,-1.04728900e-01,-5.90879321e-01,-3.81344147e-02,6.57140732e-01,-6.94689870e-01,-5.20506144e-01,-1.07452762e+00,2.29712620e-01,-5.32449782e-01,-7.10984886e-01,-5.68761230e-01,-2.48398706e-01,-9.31186378e-01,-1.07706174e-01,8.44030455e-02,-3.52953464e-01,-1.37958899e-01,-6.10627294e-01,9.60298702e-02,-7.38114536e-01,3.26203734e-01,-3.65160882e-01,-1.27716327e+00,-6.53845072e-02,-2.45364159e-01,-4.19146121e-02,-3.31007898e-01,8.31156075e-01,-1.37488693e-01,-5.36086082e-01,-9.89544392e-01,-5.78051150e-01,-4.32195455e-01,-1.49607390e-01,1.01239085e+00,5.70478559e-01,-7.87111998e-01,3.49636614e-01,-5.60164712e-02,3.60481470e-04,-4.26360995e-01,4.90510046e-01,-4.67568398e-01,6.01131856e-01,8.92858654e-02,-2.51149267e-01,-2.29489468e-02,-7.03349590e-01,8.19806233e-02,-3.48497406e-02,-1.14867054e-01,1.39540255e-01,2.66085029e-01,2.77735982e-02,-7.57234514e-01,-7.36158192e-02,-1.05199289e+00],[-0.20211929,0.45204177,0.15073241,-0.15428059,0.69280034,0.12174501,-0.2572988,0.19309661,-0.11213702,-0.42020145,-0.20569009,0.22059429,0.3914072,0.23700541,0.512938,0.5213683,-0.04397787,-0.2891785,0.11213745,-0.09914518,-0.3417891,-0.40982366,-0.30563313,-0.10657682,0.10117508,-0.01793351,0.60327023,0.07770339,0.15072806,-0.00247389,0.5046099,0.31852293,0.585395,0.15467487,0.5641468,0.15181373,0.33250007,0.1264429,0.30005738,0.24419211,0.7621222,-0.2552368,0.77666813,-0.20829216,0.08750903,0.13090909,-0.36935103,0.123563,0.12374197,0.47056997,-0.19635218,0.07282009,-0.1052056,0.14008678,0.4984735,0.15109445,-0.10914711,-0.04534596,0.10747902,0.45270222,0.16159363,0.22522849,0.32533556,-0.15626734,0.04567853,-0.68755114,-0.13670835,-0.29170272,0.5545896,0.4122467,-0.09638442,-0.41467655,-0.0385899,-0.05926019,0.16530295,0.37479544,-0.14103588,0.15341453,0.53405166,0.30617306,0.5114862,-0.16160265,0.00592509,0.35805055,-0.02122,0.16907874,0.32187167,0.06058971,0.3071545,0.16986255,0.54781944,0.5181346,-0.16661544,-0.02086124,-0.12113877,0.1627165,0.01810837,-0.34136993,-0.16057159,-0.21853437,0.3461735,0.16249426,-0.16097371,-0.04277826,-0.15857515,-0.52276146,0.35210294,-0.10150605,-0.36033583,-0.07237525,0.29630402,0.04006106,-0.11412409,0.20933148,-0.06354252,0.12637182,-0.05576422,0.03023683,-0.51870614,0.20214033,0.3072832,-0.19165799,0.22690824,0.9078564,-0.32081884,-0.8912135,0.0220401,0.2783224],[-0.25367174,0.19907038,0.2545183,-0.3241364,-1.3112664,0.10894325,0.08757643,-0.7466927,-0.02046336,-0.7557886,-0.42730084,0.22942197,0.24787521,-0.8254508,-0.26956365,-0.11495692,-0.35460123,0.00761802,-0.71836287,-1.0085032,0.2558553,-0.03247947,0.2886667,-0.17328863,0.7235314,0.18071821,-0.0800854,-0.3218541,0.2856686,-0.00320584,-0.1707714,-0.18990493,-0.4021598,0.6151509,0.47816026,-0.6751546,-1.0710229,-0.00372465,-0.20546062,-0.22319058,0.50946856,0.15893158,0.41214132,-0.22169779,-0.20538048,-0.09229328,0.42914188,-1.0074065,0.33783498,-0.09399693,0.2358132,-0.270951,0.44593987,0.0484289,-0.26576138,-0.05206618,0.02874498,-0.22131981,-0.36191082,0.05221548,0.06819887,-0.26093116,-0.6476049,0.06487562,-0.32070738,0.4152764,0.16598453,-0.4939767,0.36245584,0.18940544,-0.0347072,-0.9942906,1.1581879,-0.25849771,0.14202662,-0.2918586,-0.04850521,-0.2793822,-0.18914281,0.1589686,-0.0166937,-0.5756693,0.32363948,0.6125229,-0.8204975,-0.68163085,-0.96372056,-0.1525996,0.0618397,-0.41796073,-0.41937527,-0.13313544,0.16019139,0.66023505,-0.26598397,-0.24884401,-0.08569828,0.24360669,-0.8291889,-0.30121663,0.64601475,0.47102997,0.12059895,-0.35676742,-0.08060119,-0.9552767,0.13012528,0.19000997,-0.34447223,0.00857598,-0.7226414,0.38261446,0.3303751,-0.21366523,0.13109607,0.5315502,0.11746935,0.24923229,0.09434976,0.5262063,-0.28545934,-0.2909253,0.35951224,0.06367725,-0.42384768,-1.0885662,0.5121134,-0.10287142],[-1.5620886e-01,-9.6319862e-02,5.5944556e-01,-4.7621891e-01,1.8985520e-01,-5.3408045e-01,-3.4268290e-01,8.8556573e-02,2.7624384e-01,-3.0623097e-02,6.7176349e-03,4.4428864e-01,1.5429555e-01,-8.2276799e-02,1.5766703e-01,-4.0128836e-01,2.7113041e-01,3.0335790e-01,-2.6344562e-01,-4.2913434e-01,1.8867555e-01,-3.8486451e-02,3.6794442e-01,3.2993475e-01,3.8311213e-01,-6.1515015e-01,7.0524651e-01,-2.0647155e-01,-3.1708175e-01,4.4414029e-02,4.8561978e-01,2.7465776e-01,2.5858590e-01,8.2865573e-02,5.1453102e-01,5.9322271e-02,6.0657758e-01,-2.4741055e-01,-2.4887118e-01,2.3242202e-01,3.7533879e-01,4.9263769e-01,-1.5295567e-01,1.4937750e-01,2.4206819e-01,5.3451318e-01,-2.2794494e-01,-9.9215142e-02,-6.4811802e-01,4.6322909e-01,-2.1903361e-01,1.7733467e-01,6.1098540e-01,2.4922812e-01,-7.5025922e-01,3.5272580e-01,-3.8645920e-01,1.3793617e-01,3.3608207e-01,3.7411764e-02,4.4683427e-01,-5.6905204e-01,3.8822562e-01,4.6682808e-01,4.9337864e-01,3.5890082e-01,-1.3893762e-01,9.0035516e-01,1.7919290e-01,9.2249125e-01,-2.7439696e-01,5.9481764e-01,-7.6129520e-01,2.0396626e-01,5.5235827e-01,4.2078659e-02,4.9140760e-01,-4.4143358e-01,-8.0002636e-01,1.0788020e+00,-5.2518004e-01,-3.3261818e-01,-8.5289046e-02,-3.9073977e-01,1.6256009e-01,-8.5349001e-02,8.5420482e-02,-1.6656823e-01,-3.1043297e-01,-7.4122494e-01,6.7947350e-02,4.1723898e-01,2.5692558e-01,-3.4498945e-01,2.5970274e-01,3.3479967e-04,1.4884070e-01,6.3033193e-02,3.2513365e-01,-4.6269915e-01,5.8062696e-01,-7.5679012e-02,9.9896140e-02,8.1574345e-01,2.6010650e-01,3.8718376e-01,2.2267660e-01,-6.7373049e-01,-2.0696836e-02,-1.5453316e-01,-1.7041630e-01,3.2772854e-01,-1.9118361e-01,-1.3764277e-01,5.7160398e-03,3.5036391e-01,5.7036304e-01,-8.9016326e-02,-1.9054126e-02,5.6261301e-01,-1.4841206e-01,2.1188259e-01,-1.2622117e-01,-1.9894414e-01,2.6423395e-01,1.2003037e-01,-3.5575187e-01,1.3410184e-01],[1.27394512e-01,-1.42846978e+00,3.12546760e-01,6.20891228e-02,-1.07700706e+00,-1.34432149e+00,-1.00742757e+00,-4.98072624e-01,-1.02146164e-01,1.04339756e-01,-3.15665841e-01,3.77386332e-01,-1.12469018e+00,-1.35350749e-01,-6.89775646e-01,-3.14571947e-01,-7.45283365e-01,6.60291268e-03,-2.06030443e-01,3.59984607e-01,-7.59141088e-01,5.34752198e-02,-8.74256790e-02,-2.19805062e-01,5.80262721e-01,1.56493057e-02,-1.18202174e+00,1.44721717e-01,1.76934719e-01,-6.97265446e-01,-7.17889667e-01,1.71120897e-01,-2.15070039e-01,-8.96726906e-01,-6.82420611e-01,-5.03169239e-01,-3.46718758e-01,-1.68228522e-01,7.96007887e-02,4.68908474e-02,-5.87504745e-01,-1.42022478e-03,-6.38369560e-01,-1.18291914e-01,1.02471031e-01,-9.78395104e-01,1.05395675e-01,3.97792347e-02,2.64370412e-01,-4.83635932e-01,2.93191403e-01,-3.54731113e-01,-5.64201176e-01,-7.61431396e-01,-2.97924340e-01,-1.99612275e-01,-9.92375970e-01,-1.14405262e+00,-1.02062070e+00,-8.56105268e-01,-1.86321810e-01,-3.15513819e-01,-2.97973931e-01,8.59822035e-02,-2.39589483e-01,-2.94305205e-01,-1.06441423e-01,-2.83256292e-01,1.55645207e-01,-7.98712015e-01,-9.96860921e-01,-2.86367476e-01,-3.80531281e-01,-5.93958020e-01,-7.46880770e-01,-2.72479892e-01,5.11975527e-01,-1.02761768e-01,1.47464707e-01,1.89386029e-02,-9.75982487e-01,-5.36407292e-01,-2.76487321e-01,8.18720937e-01,-1.06160915e+00,-6.05297685e-01,-6.08395576e-01,-1.56996101e-01,-9.25990403e-01,-3.16526026e-01,-8.96737352e-02,-1.15683651e+00,2.75594592e-01,-2.07935706e-01,5.91435134e-01,5.68461299e-01,-9.26191986e-01,4.20199782e-01,8.89688432e-02,3.91036049e-02,-6.46365643e-01,-9.46134984e-01,2.19429447e-03,-5.00851333e-01,-8.00497949e-01,-2.22771004e-01,-1.17373204e+00,6.59164116e-02,1.45222783e-01,-2.49154661e-02,-4.35599029e-01,4.33751851e-01,-2.39642262e-01,4.18555848e-02,-5.81753850e-02,2.68030614e-01,1.58728406e-01,-2.54653364e-01,4.00752872e-01,7.08478749e-01,-1.17689586e+00,-3.21420521e-01,-6.39884651e-01,-1.06590104e+00,-2.95647323e-01,-2.14683890e-01,-2.05887049e-01,-2.47425526e-01],[0.31757542,0.3525849,0.17995422,0.39903897,1.520506,0.58690155,0.21073522,0.18958604,0.61537504,-0.03033256,0.2368759,-0.34842312,0.14967695,-0.20444784,-0.1399715,0.88259405,-0.13203031,0.04501228,0.06344236,0.13175325,-0.26009604,0.24150304,-0.01256182,-0.7498122,0.3085455,-0.34660378,-0.30072898,0.04325122,0.13435201,0.14669192,0.04114409,-0.08048538,-0.35094228,-0.12944557,0.3168262,-0.06972116,-0.01758887,0.0581663,0.414344,0.07633372,-0.1252812,0.27508828,-0.02157201,-0.40360066,0.10187827,-0.03395054,0.33944717,0.19910325,0.28086162,0.09867358,-0.27484986,-0.19107413,0.6119226,-0.24905561,-0.0203052,0.38426498,0.02366296,0.08702311,0.4012214,-0.09175986,-0.17110716,0.27685818,-0.4871864,0.2410038,-0.36207008,-0.04879133,0.326817,-0.37925237,-0.03063106,0.11616074,0.05727184,-0.17698275,0.0591817,-0.21621981,-0.43177056,-0.04446287,-0.35439175,0.44429016,-0.18817967,-0.01001638,0.04479533,0.29237208,-0.7342723,0.28014722,0.03751417,-0.72803617,-0.20317945,-0.24236348,0.24112529,0.17491645,-0.0480658,1.1364727,0.1766402,0.14776586,-0.45688078,-0.07780433,-0.0160664,-0.09166053,-0.35283226,-0.09089327,0.41100416,0.19388248,0.06806646,-0.3476658,-0.62954223,0.125477,0.18230066,0.5097791,-0.07854833,-0.09392335,-0.07700969,-0.5957808,-0.46593475,0.18168977,0.16866678,-0.41378352,0.3182503,-0.2755872,-0.21729952,-0.26606444,-0.19797643,-0.3898669,-0.06451194,0.00407802,0.17731912,-0.10976107,-0.12369859,0.1591443],[0.20237194,-0.4411422,-1.3284284,0.04545088,-0.65037453,0.01502455,0.5391327,0.8920827,0.21633002,-0.07621981,0.02483699,-0.86681855,0.20376806,-0.54542685,-0.00809725,0.03695581,-1.0589725,-0.67415655,-0.85536814,-0.4931294,0.33116475,-0.94124216,-0.24560966,-0.28536093,-1.3976198,-0.43483102,-0.13801974,-1.2921892,-0.06514202,-0.35090843,-0.70074433,-0.16003482,-0.22724226,-1.3272536,-0.33030078,-1.1272271,-1.2057017,0.34453434,-0.88807094,-0.3473358,-0.02744557,-0.28398907,-0.13588895,0.32262975,0.45988032,0.64936656,-1.1315733,0.62701374,-0.04227595,-0.370355,-0.28041467,0.5800358,-1.0163406,-1.4683057,-1.0587387,-0.06482382,-0.47366256,-0.4029717,-0.4335106,-0.5867429,-0.3259758,-0.27356452,-0.31108183,-0.12109166,0.55243146,-0.06040205,-0.95590657,0.06319566,-0.47803584,-0.09157294,0.7813489,-0.8953135,-0.31214416,-0.624187,-1.0311835,0.8798728,0.09303907,-0.3147554,-1.1632097,-1.2052344,-1.2599077,-0.25089687,0.27989042,-1.2509363,-0.6768885,-0.07286853,-0.79059184,-0.387401,-0.91532856,-0.5834897,0.25298914,-0.8308659,-0.63145375,-0.14456026,-0.47201332,-0.19315627,-0.08337924,0.12383739,-0.3927009,-0.06970683,-0.7494669,0.0513535,-0.10164102,-0.28106663,-0.688086,-1.1224756,-0.6502364,-0.08301333,0.6834752,0.27960166,-0.49480966,-0.610004,-0.9429616,-0.2121171,0.09144492,-0.7154583,0.05842314,-1.2474948,-0.83454335,-0.41496968,0.2075053,0.6249993,0.3354125,-0.04221552,-0.11328762,-0.69304895,0.27910614,0.14896816],[-0.17566687,-0.49477717,-0.14500058,0.07144871,0.01499052,0.11923274,-0.89913976,-0.3809708,-0.00570875,-0.91384953,0.44715816,-0.8080687,-0.31001288,-0.21387467,-0.760813,-0.50604635,-0.77441096,0.2546486,0.622518,-0.72652763,-0.10059262,-0.6517341,-0.41701886,-0.7634305,-0.67505115,-0.22566167,-0.03682553,0.16360931,-0.6012038,-0.27502027,-0.10775782,-0.08439221,-0.5958697,-0.38916475,0.08928543,-0.55579656,-0.7847448,-0.31238487,-0.52664053,-1.007771,-0.21967119,-0.2919685,-0.28326264,-0.6005217,-0.82268465,-0.64834124,0.03653376,0.01272047,-0.59832835,-0.17339028,-0.4407642,0.8602215,-0.78488535,0.41836253,-0.03970702,-0.8116254,-0.77857643,-0.2608791,0.01738012,-0.7780455,0.1399016,-0.06426633,-0.2975566,0.02540905,-0.9716693,-0.33306378,-0.7862799,-0.9590519,0.0016087,-0.5427849,-0.41511405,0.22788906,-0.8522608,0.4625539,-0.13197525,-0.2763521,0.8045871,-0.28336397,-0.16635844,-0.11829252,-0.9807562,-1.2774827,-0.42561182,-0.00135472,-0.33088687,-0.6689776,0.0045379,-0.5040528,0.1642899,-0.19543314,0.4450753,-0.26549885,0.1642252,-0.32210502,0.6425028,0.17273314,-1.0293536,-0.7530102,-0.7060114,0.04992527,-0.20701577,-0.96989393,-0.6063313,0.28284115,-0.5015988,-0.09437871,-0.22740136,-0.04869308,0.30168945,-1.3307176,-0.4308851,-0.51935375,-1.0014424,-0.4058157,-0.3263538,-0.23688672,-0.5377106,-0.73256415,0.40716565,-0.6351257,-0.19104688,0.11874582,-0.06203924,-0.4997625,-0.6568096,-0.61630905,-0.16370341,-0.61768436],[-0.02449846,-0.26763877,-1.1037546,-0.05617801,-0.89516133,0.15846679,-0.05827736,-0.1318146,0.36109227,0.42447063,-0.09987941,0.32046062,0.14235805,-0.05675947,-0.3545004,0.15926553,-0.34459364,-0.06256437,0.03775328,-0.01113784,-0.5446543,-1.1893703,-0.40428266,0.06228679,-0.721391,-0.3461562,-0.313259,-0.18790957,-0.65718096,0.5773838,0.09114918,0.07832956,1.0327345,-0.92624485,0.75504726,-0.53213197,-0.23187263,0.04181919,0.58557785,-0.13847667,0.00314796,-0.25776213,0.5788418,-0.5709151,-0.52653116,0.3348816,0.1890816,-0.16713098,0.18030372,-0.29369006,0.25868848,-0.11330053,-0.14631385,-0.8451814,-0.3567069,-0.7748478,-0.08482681,0.6034371,0.3058915,0.31855184,0.09784011,-0.29078463,-0.74801284,-0.6172562,-0.0419563,-0.22260329,0.28654405,0.34511495,0.4760551,-0.7297204,-1.0707783,-0.26611197,-0.5466379,0.07958526,-0.20405908,-0.63043,-0.14963572,0.07953621,0.11500054,-0.5787987,-0.64056313,0.23823436,0.51404727,-0.76458985,0.01607751,-0.00297682,0.5651831,-0.3057617,0.18024604,0.03132012,-0.03006741,-0.37914786,-0.1238941,-0.26358178,-0.20048292,0.14221108,-0.0846369,0.386872,0.14136067,-0.03139476,0.92923266,-0.09796341,0.28495187,-0.07435557,0.0327154,0.04124261,0.38179123,-0.36812913,-0.5760411,-0.1414166,-0.08817664,-0.7675938,0.03362694,0.34845325,-0.08437796,-0.15255558,0.41791898,-0.8518364,-0.16721608,0.31205887,-0.9341234,-0.10208127,0.19993857,0.20283979,0.4793173,0.2847408,-0.42821893,-0.18570818],[0.26467037,0.42889073,0.10189387,-0.64140743,-0.03598313,0.26442197,0.3084512,-0.33855447,-0.35670033,-0.98370403,-0.2892549,-0.21328975,-0.9547991,-0.19135132,-0.01523684,0.61927575,-0.40211162,-0.30772367,-0.54426277,-0.19934593,0.8175366,0.7792458,0.22401084,-0.16921788,0.08852308,-0.15190431,0.51263595,-0.3322463,0.01523046,0.10990702,0.29046994,-0.4985705,0.03864913,-0.26011023,-0.43624794,-0.29233378,-0.23071148,-0.49014625,-0.38526785,-0.0879366,-0.38130265,-0.2900066,0.17704743,-0.5895506,-0.3555587,-0.58185536,-0.56791115,0.35191098,-0.15209647,-0.7502857,-0.38591424,-0.3443305,-0.49787855,-0.12214678,-0.7806847,-0.23018096,-0.23286483,-0.01232327,-0.14877263,-0.59959286,-0.5507558,-0.6263205,0.1233485,-0.4213725,-0.09748395,-0.26083714,-0.5415297,-0.11358526,0.01299628,-0.29497057,0.03591419,0.17588958,-0.8811438,-0.36831665,-0.74897,-0.50651175,-0.174228,-0.33686072,-0.37389547,-0.18717648,0.712874,-0.47390944,-1.5426946,0.13989024,-0.6807282,-0.29372403,-0.6783417,0.09162499,0.4030355,-0.8469926,0.12003546,-0.86046326,0.0303868,-0.7164457,-0.40250155,0.44699484,-0.27522793,-0.49372244,-0.71398187,0.13358417,-0.15807018,-0.9906905,-0.13821168,-1.4712613,-0.07318992,0.13458186,-0.02933921,-0.3001299,-0.00824259,-0.11280628,-0.42850867,-0.3928461,-0.16957738,0.00200076,-0.74184966,-0.9690374,0.01436921,0.1631343,-0.48059168,0.24923946,-0.90709776,-0.7694926,0.05866329,-0.4737708,-0.1945533,0.28384298,-1.8389212,-0.46118242],[-1.41501829e-01,1.63969904e-01,6.31822050e-01,-1.39723361e-01,-2.27569610e-01,5.57723522e-01,-3.43551226e-02,-6.86845481e-02,-4.49063420e-01,-2.55472094e-01,1.06685571e-01,-3.81320775e-01,-1.86967239e-01,-1.94527656e-01,2.29919478e-01,-1.25762165e+00,5.59695065e-01,-1.93209961e-01,2.23293647e-01,-1.24223672e-01,6.58946633e-01,2.64431268e-01,-2.77261347e-01,2.02739406e-02,-1.62960410e-01,-7.97828585e-02,6.46244466e-01,2.47384101e-01,-9.35888616e-04,-7.87308395e-01,1.96977735e-01,-1.20595491e+00,-5.78266792e-02,-4.80552107e-01,-6.24628544e-01,-6.15019858e-01,1.62816092e-01,7.02618808e-02,-8.93992066e-01,-3.11395019e-01,1.45542949e-01,6.06549494e-02,-3.18106443e-01,7.92888403e-01,-9.08806145e-01,-1.00144029e+00,3.80308717e-01,-2.40370169e-01,-1.18724859e+00,1.22104667e-01,-7.39994049e-01,1.06770419e-01,-5.36553741e-01,1.09031484e-01,-7.28016853e-01,-1.12697208e+00,3.35026264e-01,4.04853165e-01,-5.46795905e-01,-7.74044573e-01,-2.19913915e-01,5.67443371e-01,-6.08287156e-01,1.67414382e-01,2.62502953e-02,-1.93252623e-01,-1.95592850e-01,-8.35373998e-02,1.43696263e-01,-5.59119880e-01,-6.45734072e-01,-6.49272978e-01,-6.50014222e-01,-3.64949882e-01,-1.02329206e+00,5.99891581e-02,-3.99602920e-01,-5.03510237e-02,-4.23026472e-01,1.56034797e-01,-8.80086899e-01,-3.36599022e-01,-1.33138210e-01,-1.47708133e-01,9.70265418e-02,-4.12741721e-01,1.39574841e-01,-1.42961759e-02,3.05367596e-02,-4.62749451e-01,2.26895273e-01,-2.29623497e-01,1.03642941e+00,-6.72053397e-01,-1.87017903e-01,9.22539011e-02,-8.48530233e-01,2.97529519e-01,-6.57867789e-01,2.10478753e-01,-9.15057063e-01,-3.84713802e-03,-7.47572482e-01,1.41852470e-02,-7.18033314e-01,-3.59539390e-02,-4.75291669e-01,-1.46768227e-01,-6.32143378e-01,-4.47657198e-01,-7.93361902e-01,-3.28754514e-01,-1.46780118e-01,-7.03972101e-01,-1.05684362e-02,-1.15021365e-02,-1.28342962e+00,-1.31247908e-01,-1.53141308e+00,-8.54616284e-01,1.19086750e-01,-1.02323055e+00,-4.48460549e-01,-3.23251379e-03,-6.14260733e-01,3.15261513e-01,-2.14499831e-01,-1.16703771e-01],[-1.08652689e-01,-1.69689864e-01,-3.84330958e-01,-2.08751246e-01,-5.11231363e-01,-4.95879292e-01,-2.26324528e-01,-5.84821582e-01,-1.05990745e-01,7.53183484e-01,3.46362352e-01,-1.58689117e+00,-1.99314371e-01,-6.45222008e-01,-1.02349329e+00,-4.86234486e-01,7.71525279e-02,-1.31428027e+00,-9.72980380e-01,-1.40194222e-01,-3.49694222e-01,-4.26462531e-01,4.77189004e-01,-2.93171436e-01,-2.08245724e-01,-1.01322651e+00,-2.37524286e-01,-2.10051477e-01,-1.26899570e-01,1.53962120e-01,-6.48674548e-01,4.36386466e-01,-8.94319415e-01,-3.05743128e-01,-2.65506566e-01,-6.83921456e-01,-5.84619641e-01,1.60546735e-01,-7.49495327e-01,-1.34513512e-01,-7.91730046e-01,4.03703809e-01,-6.61158025e-01,-9.08729315e-01,3.60472083e-01,2.80821472e-01,-5.55823982e-01,1.17313378e-01,3.69157135e-01,-6.40595973e-01,-4.76113379e-01,4.68069941e-01,-5.21800220e-01,-3.02080274e-01,-2.32834101e-01,5.81031322e-01,-2.97796220e-01,-9.77447256e-02,-7.11304843e-01,6.45532447e-04,-9.66263562e-02,-8.46599042e-02,3.44567150e-01,2.76507527e-01,-3.59591454e-01,-8.05825531e-01,-5.12846708e-01,-4.18185979e-01,6.86037466e-02,-4.85868663e-01,8.25625002e-01,-5.40752769e-01,-8.71608630e-02,3.74783635e-01,-6.72955438e-02,-1.43858148e-02,2.07189217e-01,-6.18350089e-01,-1.75880641e-01,6.36569381e-01,-6.05098426e-01,-6.11233413e-01,-3.59743834e-01,-1.44824040e+00,-1.20294444e-01,4.17620838e-01,-4.70201910e-01,-1.12897956e+00,-4.59167659e-01,-1.15780640e+00,-6.57609284e-01,-5.86829901e-01,-3.93690407e-01,5.69777377e-02,-9.93800461e-01,-1.95357010e-01,-2.51681507e-01,2.81783700e-01,-1.00156689e+00,-2.82864064e-01,-3.86917621e-01,6.16742432e-01,4.88125801e-01,2.42484421e-01,-2.39852175e-01,-5.44856608e-01,-5.53787589e-01,-8.02074075e-01,-7.93451890e-02,4.53318395e-02,-2.32929140e-01,-7.46990681e-01,-5.20169556e-01,-1.25394240e-01,1.21057227e-01,-6.83294833e-01,-4.76913422e-01,7.61116296e-02,-6.40172958e-01,-8.21444988e-01,-6.56026065e-01,-1.28905490e-01,-2.80312747e-01,-3.13915461e-01,-5.93634963e-01,-6.63487494e-01,-7.41181493e-01,-6.28471911e-01],[-0.1597687,0.4235841,0.15744935,-0.27664235,-0.69149214,-0.21301873,-1.2120017,0.34313354,-1.5648618,-0.39805532,0.37598202,0.22006136,-0.647948,0.55240583,0.14001115,-0.1012399,0.3669945,-0.35141253,0.16788118,-0.020458,-0.5730106,-0.25444195,-0.27307302,-0.15530445,-0.6199582,-0.753257,0.52014023,-0.6427712,-0.38886616,-0.35748616,0.8565203,-0.62762946,-0.42111588,0.10206624,0.3352425,-1.0010694,-0.7348621,0.8678873,-0.749542,0.6556906,-1.3742524,-0.26055583,-1.1181551,-0.44822794,0.20667234,0.00646876,-1.3472708,0.22688545,0.09082918,-0.43649644,-0.6177252,0.07210412,0.6338288,0.01579476,0.6704892,-0.7669456,-0.08346427,-0.0112759,-0.7541595,-0.40354028,-0.3161951,-0.47295192,-0.13962305,-0.8612257,-1.8136278,-1.2125987,-1.2274183,-0.51507926,0.12349857,-1.3787256,-1.2147676,-0.81943166,-0.4302479,0.05406964,-0.0612538,0.05730944,0.03704526,-0.3635256,-0.31592074,-0.34954795,-0.1288482,-0.4940862,0.09885065,-0.12687248,-0.5073282,0.05748049,-0.08989101,0.8432603,-0.06521957,0.4173415,0.0743584,-0.23428245,-0.9635977,-0.8154413,-0.42697895,0.14269544,0.02257883,0.46577454,-0.9700056,0.3215067,-1.3869535,-0.23590162,0.37907723,0.2729351,-1.6942586,-1.2853488,0.09902622,0.13044976,-0.13153842,0.43714038,-0.6622466,-0.19905226,-0.39172184,-0.8814659,0.36549795,0.22632885,0.46919116,-1.0731075,-1.0584705,-0.8619422,-0.08220468,-0.44002426,-0.8565137,-0.09334273,-1.0079113,0.19379574,-0.13845177,-0.9509339],[-0.41025802,-0.3695605,0.39777583,0.03415797,-1.0949686,-0.19735043,0.31994474,0.1770901,-0.22490664,0.2660365,-0.09704245,0.42562082,-0.02340729,0.07259025,-0.18539582,-0.28404096,0.18016092,-0.00756534,-0.41335404,0.25112405,-0.59563184,0.7663713,0.9958078,-0.12029414,-0.6060038,-0.34800503,-0.26903358,0.18725166,-0.580954,0.9855598,0.14696772,0.69677556,0.01041722,-0.4667222,-0.16913983,-0.05127412,-1.2339518,-0.5449713,0.0834692,-1.084206,0.09834172,0.18749925,-0.8661511,0.28943938,0.05544771,-0.7329439,-0.39155126,0.00393156,-0.39671272,-0.41000226,0.4555022,-0.8237007,-0.8023816,0.2809969,-0.5614067,-0.25985363,-0.5797603,-0.01447523,-0.04101962,-0.20588665,-0.27345958,0.4425241,-0.01292967,-0.30787483,-0.9685145,0.909323,-0.31363878,0.60395086,-0.16486703,0.06361315,-0.4168732,0.4339509,-0.70524395,0.25884795,0.08545896,0.35656548,-0.2142292,0.45087293,-0.53541976,0.25927398,-0.12816541,-0.4715203,0.03958023,-0.0908692,-0.31942818,-0.2014202,0.0788753,0.23358196,-0.3326712,-0.5816437,-0.8803749,-0.2301487,0.51524943,0.53270394,-0.72665495,-0.54769385,-1.0258862,0.36428225,-0.55355686,0.11007801,-0.8415287,0.04496612,0.00550276,-0.1168308,-0.44095117,0.20500197,-0.70295596,-0.6174651,0.20388414,-0.10929841,-0.45054016,-0.4320232,0.11421722,-0.3379342,0.20664863,-0.11406696,0.17984974,-0.17518322,-0.01807883,-1.2066518,-0.08845626,-0.6023152,-0.17625521,-0.6148314,0.07573497,-0.87371266,-0.31281212,-0.9324224],[-0.7304254,-1.0563172,-0.09227949,-0.91568947,0.01464317,0.28660733,-0.7616076,0.20359157,0.04136146,-0.9188709,-0.2906925,-0.28077587,-1.0622044,-0.1983982,-0.35375857,-0.06142784,-0.04177004,1.143065,-1.3602712,-0.53123105,0.02136904,0.2394565,0.31317696,-0.17378098,-0.37857884,-0.872156,-0.21952513,-0.1797898,-0.3683899,-0.31897083,-0.32965335,-0.1951588,-0.23935927,-0.15692815,0.05623318,0.00615929,-0.9984353,-0.51237136,-0.18542922,0.37395924,0.06627579,-0.01405773,0.3108892,-0.11228999,-0.13507257,-0.5648946,-1.1612345,-0.2614195,-0.54548615,-0.3404191,-0.38659978,0.442074,0.19421135,-0.45100677,-0.2704552,-0.2385739,0.5510954,-0.24608906,0.18404832,-0.49703732,-0.16643254,-1.0017414,-0.48839587,-0.46990168,-0.06372608,-0.6335471,-0.27298057,-0.09893517,0.26872745,-0.3569532,-0.5767074,-0.25714004,0.38657627,-0.9947078,-0.38171917,-1.3751663,-0.3036208,-0.50479794,-0.6781709,-0.05913234,0.01910576,-0.26640898,-2.353521,-0.1479698,-0.42681777,0.08101302,-1.3644564,-0.38168338,0.23960075,-0.1519265,-0.17393579,0.12305122,-0.5268619,-0.6651592,0.10244717,-0.2447162,-0.50574636,-0.15731138,-1.5413908,-0.8459073,-0.24857129,-0.8848239,-0.70518935,-0.9586445,-0.10554983,-0.73821735,-0.39538214,-0.32509992,-0.42297405,-0.15602788,-0.7705495,-1.1833413,0.07611522,-0.4026868,-0.57943594,-0.30434003,-0.25987428,-0.2080373,-0.41398305,-0.35995254,-3.1101613,0.3795191,-0.41236627,-0.28982058,-0.38928545,0.2742912,-1.5023552,-0.66050273],[1.11968659e-01,-2.74466395e-01,-3.49927843e-01,-6.81615651e-01,-4.40178931e-01,-6.58287778e-02,-3.05451155e-01,5.88097908e-02,2.37869680e-01,5.21201015e-01,4.69997555e-01,4.67527986e-01,-3.64064842e-01,3.36742699e-01,1.87434703e-01,-4.41663444e-01,8.13386515e-02,-1.30049562e+00,-3.89586926e-01,-2.55815685e-01,6.14592910e-01,7.25136340e-01,1.32835060e-01,-1.48816288e-01,-4.04766858e-01,-1.08713359e-02,-2.95199066e-01,8.99910450e-01,3.39235425e-01,5.00744700e-01,-1.04002357e+00,-2.42846176e-01,5.51554784e-02,1.47054747e-01,1.41566902e-01,1.42454907e-01,-3.69897664e-01,4.06948850e-02,-4.62941557e-01,-4.02854681e-01,2.86858410e-01,-4.05025482e-01,-2.09108561e-01,5.13714910e-01,-6.61358237e-01,-3.94969046e-01,-3.02158266e-01,-1.98763505e-01,6.29197299e-01,-2.65672147e-01,-3.05888891e-01,-5.55579245e-01,-2.55618960e-01,-1.75839230e-01,-2.86261380e-01,3.45540166e-01,1.45169586e-01,-2.90007144e-01,-6.13876998e-01,-6.66612983e-01,-6.41970575e-01,-1.75197780e-01,5.10031521e-01,2.07372919e-01,1.16510756e-01,-2.76924014e-01,1.70644507e-01,-2.51366198e-01,3.74730796e-01,3.19513679e-01,7.30857790e-01,-2.59279996e-01,-3.84429783e-01,7.30017662e-01,-7.73546770e-02,4.47352156e-02,-7.59369060e-02,-2.12109998e-01,-3.79902512e-01,5.12265693e-03,1.13484720e-02,6.31421208e-02,5.89787006e-01,-2.56348044e-01,6.54383004e-01,1.70921877e-01,5.10518610e-01,-4.77746874e-01,-1.63824230e-01,-1.07597925e-01,-2.97152936e-01,2.21515030e-01,-9.64529872e-01,2.91359395e-01,-1.47907123e-01,9.38107027e-04,1.09701611e-01,-8.11975673e-02,1.23189002e-01,-5.86735785e-01,-3.52680624e-01,-3.08786094e-01,5.98884001e-02,1.32606357e-01,5.40129721e-01,2.91519374e-01,9.01022702e-02,-2.77584851e-01,-2.78222173e-01,-7.75143206e-02,-4.79269847e-02,-9.80767384e-02,-2.79843628e-01,2.10670739e-01,-6.11802697e-01,1.74891382e-01,2.91084290e-01,8.40536714e-01,-5.95503747e-01,-4.14212734e-01,3.66118729e-01,-6.03826344e-01,3.99433136e-01,2.90292855e-02,2.28095457e-01,-7.63516009e-01,1.30099013e-01,5.35155475e-01],[0.67008656,-0.1747267,-0.65298045,-0.5709963,-0.29409522,-0.24778643,-0.8568788,-0.40053174,0.27581087,0.20706387,-0.8890342,-0.916051,0.0442439,-1.5551176,-1.58975,-0.47462246,0.5616388,-1.3173261,-0.67458254,0.09821309,-0.6193034,-0.28801626,-1.0001972,0.2889968,-0.07711112,-0.5438183,-0.9283054,-1.399273,-0.54970115,-0.7837233,-0.72173977,-0.89395225,-0.33114833,-0.5974811,-0.30232626,-0.67420083,-1.0443876,-1.5213251,-0.7521041,-0.03999954,-1.0415244,0.48317283,-1.3340138,-0.96521187,-0.15886182,-1.4867669,-0.48387486,-0.77217674,-0.82474905,-0.16106188,-0.5135627,-0.90241534,-0.87622756,0.47757682,-0.67825174,0.16713822,-0.5647544,0.8691844,-0.33697042,-0.21684419,-0.7217301,-0.29794815,-0.25497383,0.092819,-0.9001085,-0.73868096,-0.01892238,-0.78474826,-0.02684191,-0.55958754,-0.4480326,-0.1680763,-0.17473777,-0.996812,0.63206583,-1.3278518,-1.0582438,-0.9761481,-1.0758355,-1.4635491,-1.0478135,-0.6840963,-0.29163134,-1.3879136,-0.3270132,0.50343823,-0.01047596,-0.8082178,-0.68943465,-0.6582788,0.0291814,-0.23436671,-1.1503161,0.25988746,0.90130454,-1.0562527,0.18746659,0.09514348,0.13818836,0.14457066,-0.50584465,-0.9536254,-0.2544604,0.07118174,0.8396104,-1.0668658,-0.34671983,-1.3318286,-0.70889056,-0.11393415,-0.54869545,-0.3533547,0.33788592,0.29637676,-0.3199754,-1.2986288,-0.27136472,-1.4990522,-0.5864893,-0.8404753,-1.0851872,-0.22924112,-0.62789935,-0.12581147,-0.35101053,-0.43722647,-0.34320247,-1.208044],[-0.41853565,0.45862135,0.00564115,-0.42869866,-0.6270887,-0.66846806,0.6489119,0.0322268,-0.21306925,0.45198485,-0.3617062,-0.12686546,0.2233669,-0.51901823,0.11587953,-0.51207286,-0.7026282,-0.72753906,0.5989893,-0.05097914,0.33495313,-0.65321636,-0.29959464,-0.5706703,-0.3830476,-1.2382265,-0.35499522,0.27924058,-0.7878618,-0.0648038,-0.26278394,-0.2312857,0.34629717,0.04708359,-0.67687565,0.26411024,-0.16056061,-0.53531355,-0.07889229,-0.16887605,0.53501517,-1.2738526,-0.6111592,0.6145691,-1.0537257,0.19775654,0.3838935,-0.8386009,-0.04835049,-1.1765785,-0.69449943,-0.28639233,-0.02059039,-0.21257667,-0.3237975,0.39088252,0.18525514,-0.9308241,0.20681377,-0.481012,-0.29112706,-0.01794309,0.6218439,-0.81911236,-0.691508,-0.31194088,0.01259525,-0.61744463,0.43018645,-0.9157769,-0.73625845,-1.3330508,-0.313116,-0.6380633,-0.52182335,-0.50055265,-0.7066267,-0.2508268,-0.3752625,-0.40973723,0.15892546,0.04196744,0.1405188,0.28191143,-0.30578485,0.45288134,-0.95482695,0.44620955,0.31836528,-0.49336144,0.06838835,-0.31048337,-0.53302735,-0.5364753,0.06787569,-0.4856292,-0.11691672,0.03979029,-0.47600293,0.00484473,-0.74288064,0.3880936,0.427059,-0.45904863,-0.49919134,-0.7757243,-0.51229894,-0.97088706,0.0874093,0.08723411,-0.77535826,-0.73540294,-0.531669,-0.6186374,0.13139878,0.15380254,-0.17574719,-0.1424084,-1.0598661,-0.8857707,-0.4625575,-0.32869333,-1.1562157,-0.29966742,0.26684776,-0.49610746,-0.578158,0.15789631],[-1.2805538,-0.50981605,-0.4213546,0.24099228,-0.42941707,-0.4814192,0.22097905,-0.37948534,0.3061119,-0.37935403,-0.9496816,0.17093344,0.48330706,-1.1385692,-0.6033838,-1.0379862,-0.6456246,-0.85387605,-0.1822764,0.3811293,-0.06232688,-0.95551884,-1.1105772,-0.04661982,-1.1791533,-1.0216115,-0.42958826,-1.3079352,-0.37558624,-0.8696288,-0.8846831,-0.36341763,0.9779587,-0.5632958,-0.3523218,-1.0115402,-1.2748401,0.18496679,-0.34920102,-0.10854402,-0.11903185,0.00768288,0.10479167,0.28989032,-0.29035267,-0.32699993,-0.8111628,-0.6749839,-0.836664,0.18711752,0.00454838,0.08401133,-0.1455421,0.2682959,-0.7841198,-0.93537235,-0.30091664,-0.7332156,-0.5942039,-0.43854707,-0.1612681,1.0841677,-0.9472677,-0.5261203,-0.08053671,-0.6820725,-0.8678215,-0.05842753,0.05529012,0.5452418,0.02925183,-0.18842241,-0.03568748,-0.18488376,-1.0335418,-0.38384545,-0.01985337,-1.0685241,-0.7234211,-0.97566813,-0.5409844,-0.77318436,0.24638812,-1.2137002,-0.31518722,-0.23083901,-1.0083008,-0.7745503,0.13446924,0.11295269,-0.5928438,-0.50925255,-0.3320583,0.5499863,-0.7147902,-0.41035414,-0.7406472,-0.30227765,-0.7646848,0.38161156,-0.7151638,-0.45796087,-0.9664822,0.03049303,-0.18286912,0.36338356,-1.1852156,0.21397302,0.28535908,0.20313549,-0.0656275,-0.64106864,-0.40591416,-0.56264794,-0.54613537,-0.58998245,-0.26563823,-0.74991965,-0.9983651,-0.7119176,-0.77898353,-0.07823291,0.10184174,-0.28981858,-0.6508019,0.04952281,0.14888962,-0.24941857],[0.37013844,0.2871628,0.10240669,-0.14153078,-0.6315334,-0.43374303,-0.06365793,-0.32897702,0.06459858,-1.1058408,-0.6330785,0.45166895,-0.15242672,-0.33944574,-0.65293974,0.42080414,0.6076832,-0.45927033,-0.6257878,-0.07776708,0.44624826,0.7630077,0.17988782,-0.20041832,0.42420036,0.17283124,-0.00578545,-0.8604527,-0.21532802,-0.4652018,0.598051,0.3851733,-1.0037624,0.03377642,-0.4287767,-0.6242212,-0.38735384,-1.8343239,-1.0685382,0.1428242,0.25487292,0.33640477,-0.14000212,-0.51266456,-0.14077324,-0.19639002,-0.4513404,-0.07104916,-0.39088342,-0.30600286,0.0728531,0.5188557,0.55311155,0.16643596,-1.0678859,-0.5935854,-0.0054158,-0.94607496,-0.11350951,0.14864537,-0.29699925,-0.49631092,-0.06386728,0.22278367,-0.13783649,-0.98702526,0.3215336,0.3123574,0.49082628,-0.40815914,-1.1413059,0.30173394,-0.7200844,-0.31826434,-0.2524223,-0.2424083,-0.1278694,0.20859738,-0.24816659,0.5274139,0.6599082,0.13994543,-0.48657998,-0.1964454,0.01482144,0.33308497,-0.368202,-0.1634503,-0.572797,0.17579727,-0.6547937,-0.1814664,0.40675315,-1.1096979,-0.02003505,0.30653208,0.12778044,-0.271983,0.23085542,0.05341512,-0.17897359,-0.11463904,-0.4054033,-0.34944373,0.31489828,0.17735781,-0.49519148,-0.05612405,-0.0754488,-0.36374134,-0.4358666,-0.28740925,0.04669071,-0.20657197,-0.4717361,0.05512229,0.09448855,-0.17444383,-0.84359926,-0.0574173,-0.6163672,0.54285544,-0.45913664,0.32902095,0.38891134,0.510993,-0.4668875,-0.10317333],[-6.38138533e-01,2.69057900e-01,-6.16460264e-01,-2.29760647e-01,-2.60743618e-01,-2.71175891e-01,-3.52619976e-01,-1.97546467e-01,-3.02894056e-01,2.38732859e-01,-5.14273159e-02,2.24490210e-01,-2.25094780e-01,-1.99233200e-02,-6.40301585e-01,3.01296234e-01,-4.20432240e-01,-1.01050675e+00,3.83658111e-02,6.10755868e-02,-3.45467836e-01,-3.30314517e-01,-2.71524370e-01,-3.44065502e-02,1.64286032e-01,-4.23907697e-01,-5.69976866e-01,-7.63582051e-01,2.35139474e-01,-2.02545896e-01,6.23354137e-01,-5.11004627e-01,-6.86853528e-01,-7.87279367e-01,-9.85661864e-01,-2.63427049e-01,2.27713995e-02,3.22399467e-01,4.65570688e-01,-1.95023179e-01,-4.16255385e-01,1.02390654e-01,-2.30903059e-01,2.37283573e-01,1.97929963e-01,-4.92588311e-01,-2.79479057e-01,1.63207948e-01,3.13804328e-01,-1.81747794e-01,2.91305333e-01,-5.64257018e-02,4.43607979e-02,-3.56304586e-01,4.41362470e-01,1.38728097e-01,-1.04138277e-01,2.64615327e-01,1.82378888e-01,-1.93218470e-01,4.92986143e-01,-6.87264740e-01,-9.71867621e-01,-2.27481246e-01,-1.75020665e-01,-1.18941627e-01,4.58281100e-01,-3.25013757e-01,-4.29587625e-02,-3.64243053e-02,3.13505620e-01,-3.03908736e-01,-4.34789881e-02,5.49777448e-01,-1.65870652e-01,-7.96695352e-01,1.98397890e-01,-2.49934644e-01,3.97354364e-01,3.07462156e-01,-3.32881100e-02,2.51740634e-01,2.69028574e-01,-2.57753521e-01,3.25947255e-01,-1.19587868e-01,-4.82669473e-01,1.05694421e-01,-4.10214156e-01,7.80226737e-02,2.42805168e-01,-6.13142252e-01,2.73368031e-01,-4.88680661e-01,-3.10327709e-01,-7.60222256e-01,4.04211372e-01,-1.32958889e-01,-2.49340758e-01,-8.17806274e-02,-6.68491900e-01,-1.80382573e-04,-4.73940559e-02,-7.93424308e-01,6.34395957e-01,1.33447930e-01,-7.14591742e-01,-2.36988142e-02,-1.10263205e+00,4.40792471e-01,-1.29909232e-01,-8.62780273e-01,-3.06515545e-01,1.98526904e-01,6.17778450e-02,-7.53605247e-01,4.76711720e-01,-2.48764679e-01,-7.00165331e-01,4.77098763e-01,-2.41543010e-01,-3.37787360e-01,-3.97028297e-01,1.68463811e-01,5.93812644e-01,-4.97579068e-01,-2.70136535e-01,-5.42903364e-01],[-0.4119607,-0.85912734,-0.3679473,-0.40656045,0.32701257,0.45363718,-0.28802732,-0.35429183,-0.79319453,-0.20912763,0.07684996,-0.29941988,0.08282613,0.38841024,-0.5917883,-0.67173254,-0.4762847,-0.227031,-0.12207846,-0.3569349,0.13144654,-0.03721399,0.03011324,-0.4078535,-0.8330808,0.4413037,0.3136433,0.27667013,-0.49425516,0.58200073,-0.47588232,-0.1236058,0.02849592,-0.14361621,-0.57663417,-0.4738402,-0.5411798,-0.09717543,0.38230455,-0.45221958,0.07430398,-0.38638234,0.281439,-0.15473346,-0.59231955,-0.49474874,0.5487041,-0.47929823,-0.1488662,-0.71545446,-0.19787747,-0.29686567,0.35722446,0.09913548,0.03315024,-0.25071844,0.1004323,-0.63911134,-0.6687962,0.04939231,0.35153913,0.30752668,-0.05129053,-1.1227188,-0.23637696,-0.34760338,-0.5625366,-0.16330883,-0.19788241,-0.41467303,-0.10021793,0.07902472,0.17675501,0.06015247,0.12391252,-0.5012357,-0.63534397,-0.2701976,0.4538248,0.06607626,0.35531244,0.20373224,0.4194475,-0.20016246,-0.14653026,-0.26670077,-0.08789884,-0.43482068,0.08986861,0.450111,-0.41255793,-0.279546,-0.15191391,-0.5622519,-0.06821319,-0.21123658,-0.7568555,-0.06107718,0.38789162,-0.8557371,-0.10484534,-0.8465624,-0.16336876,0.1134169,-0.39175025,-0.5803958,0.3190529,-0.05235973,-0.13700081,-0.5677133,-0.04690002,-0.15387227,-0.27602956,0.07365436,-0.02765352,0.2855461,-0.29999134,-0.46589658,0.06622826,-0.3259795,-0.04906772,0.21604063,-0.8101297,-0.18466103,-0.64625806,-0.7301145,-0.22136524,-0.04617108],[-0.28003636,0.37558094,0.602903,0.08950213,-0.17462151,-0.0505728,-1.114895,0.49990442,-0.03708715,-0.44503075,0.05171709,-0.04553104,-0.2690384,0.21731819,0.00387365,-0.31946516,0.3408317,0.6820787,0.28602976,-0.21726376,-1.3260088,0.8163555,-0.06476154,0.43833825,0.17286003,0.1993481,-0.01031612,-0.03054838,-0.8287508,-1.1079367,0.3019083,-0.10122492,-0.466364,0.09158514,0.02706769,-0.09648458,0.2971062,0.18899223,-0.29306912,-0.40037635,-0.97407407,0.31371164,-1.1224879,-0.11076176,0.21053882,-0.03522778,-0.18852903,0.5746296,-0.76471925,-0.23892975,-0.90507525,0.32519218,-0.8948084,-0.03208713,-0.36605045,-0.03971042,0.17697068,0.03198201,-0.02209089,-0.76816356,-0.30399236,-1.1333201,-1.0648036,-0.19550823,-0.6795889,-0.9800768,-0.54285884,-0.4061738,-0.10860468,-0.06862659,-0.929814,-0.5578045,-0.20834272,-0.42901385,0.12296893,-0.76892185,-0.46926072,0.01690424,-0.11510843,-0.0423941,0.48561066,0.06337112,-0.96088254,0.17471656,0.38453448,-0.17921564,-0.52699775,0.03950476,0.49523354,0.2986871,0.30901557,-0.09265838,0.21851036,-0.50997734,-0.08705161,0.30751976,0.34065402,-0.37228495,-1.2118349,-0.05562157,-1.2173616,-0.18859676,-0.5006426,-0.5010817,-0.38314748,-0.19853683,0.80510306,0.07519972,0.31575027,-0.13470413,-0.53843606,-0.59288746,0.36824265,0.0979901,-0.38129702,0.05728962,0.25325528,-0.22069535,-0.84970206,-0.26474077,-0.80148304,-0.31241694,-0.4934107,0.04892631,-0.9782432,0.08342619,-0.9588409,0.03011899],[-0.30054232,0.35830387,-0.11649583,-0.11512766,-0.28775617,-0.03141493,0.20755708,-0.760879,0.23142815,-0.2251035,0.20130463,0.48970872,0.21361548,-0.38839355,-0.01535383,-0.7890402,-0.5446624,0.24446304,-0.6080438,-1.9429157,0.11602423,0.28180686,-0.6151199,-0.8946503,-0.02228648,0.7580078,0.0913069,-0.6733784,0.34328756,-0.7457911,0.09484111,0.27154076,0.33096954,-0.45380613,0.18200211,-0.37300304,-0.41658592,0.4685493,-0.01654905,-0.19997096,-0.47232196,0.0677601,-0.27240995,-0.43922046,-0.11999942,-1.089283,-0.14840616,-0.6634205,-0.04460376,-0.07326266,-0.2126708,-0.06384744,0.83270067,-1.0096695,-0.0468895,-0.7127239,-0.3476732,-0.02282414,0.03721593,0.39564452,-0.5989938,-0.41482493,-0.37812123,-0.25683385,0.16691072,-0.02777403,-0.03427157,0.01937843,-0.41379777,-0.6541102,-0.06000432,-0.30186057,0.60223556,-0.51340455,0.34535977,0.00301158,-0.9296926,0.16534445,0.14956696,0.17023818,0.57470256,-0.14331177,0.32773444,-0.4540547,-0.75031126,-0.25094584,-0.1625027,-0.35066423,0.31583345,0.17642939,0.2968698,-0.35668942,-0.09810012,-0.07865024,-0.3268172,0.26155126,-0.704833,0.5455022,0.30748713,-0.40392157,-0.13751772,-0.60873365,-0.10927329,-0.0393049,0.17736875,0.3176114,-0.13533226,0.02111595,0.3223441,0.06881443,-0.80978215,0.4648746,0.40221876,0.05388853,-0.1676252,0.38703763,0.00890367,-0.5133738,-0.19467686,-0.07756721,-0.3505348,-0.72384316,0.53568566,-0.01831913,-0.02869141,-0.03901365,0.20662305,-0.1543519],[-1.0690075e+00,-6.4671630e-01,-1.0621896e+00,-3.0453438e-01,-4.8906589e-01,6.9463086e-01,-7.6289183e-01,-2.3510319e-01,-1.2569863e+00,-6.2142670e-01,-1.0189837e+00,-4.4265994e-01,-5.2416563e-01,4.2400557e-01,1.6989049e-01,6.8904042e-01,4.7404128e-01,-5.4805607e-01,-9.9399394e-01,1.6616470e-01,-4.0726990e-01,-1.0472356e-01,-4.2701700e-01,1.6436841e-01,-1.3672061e-01,-5.5058128e-01,1.2589501e-01,-2.4761593e-01,-3.2115926e-04,-1.1542287e+00,1.4252171e-01,-2.1277806e-01,2.4244171e-01,-9.2204565e-01,8.1409615e-01,-2.6186951e-03,-8.5564047e-01,-2.7672103e-02,-3.8656536e-01,1.7308578e-02,-4.1025695e-01,-6.2068809e-02,-1.6072635e-01,3.4770134e-01,3.8487878e-02,-3.8993168e-01,-3.1381361e-02,-4.0052640e-01,-5.0297403e-01,-1.8928394e-02,-1.6084534e+00,-5.9750801e-01,-9.7476594e-02,8.8625515e-01,3.1121892e-01,1.2669191e-01,3.7821978e-01,-6.8001258e-01,-9.8923659e-01,6.0441263e-02,-7.5776392e-01,-1.5414108e-01,5.8483083e-02,-9.2809892e-01,-4.6224451e-01,-6.0706520e-01,-1.1953566e+00,-5.5065650e-01,2.4117889e-01,-7.2849482e-01,-8.4697759e-01,9.2777796e-02,-5.1007950e-01,5.2315629e-01,4.6731648e-01,1.5478128e-01,4.5619747e-01,3.2415412e-02,4.4315502e-01,-2.8576398e-01,-3.4560543e-01,-5.0592387e-01,1.3988341e-01,-5.4483518e-02,-1.0123146e+00,2.2836377e-01,1.5351713e-01,1.4112210e-02,-3.5968563e-01,4.7044244e-01,1.7576812e-01,-5.0489396e-01,4.2474672e-01,-1.4054538e-01,3.2521060e-01,-6.8805844e-02,-8.3448112e-01,-2.7001682e-01,-3.0697820e-01,-3.5395585e-02,-7.8418630e-01,-2.7300638e-01,5.1266867e-01,-5.7038404e-03,-2.1455545e-02,-1.8444197e-01,-3.4148180e-01,-1.6937813e-01,5.4875255e-01,-7.6687865e-02,-8.2927346e-02,-5.4754961e-01,-3.9431134e-01,1.9130910e-02,3.5903898e-01,3.7651396e-01,-2.7011350e-01,-9.7399193e-01,-1.0276340e+00,-1.1075405e+00,-1.5253880e+00,1.0410936e+00,4.8230496e-01,-1.0702718e-01,-1.1620840e+00,-6.0998774e-01,4.8123854e-01,-5.6445062e-01],[-1.90286845e-01,-8.68914425e-02,-2.96680868e-01,-4.84283835e-01,2.03615472e-01,-5.07436574e-01,-7.03805864e-01,-9.07852888e-01,1.08697759e-02,4.28692669e-01,-5.39837122e-01,-2.34050587e-01,1.60543025e-01,2.44554281e-01,1.90623701e-02,-3.85080315e-02,-6.98834419e-01,2.64703780e-01,-2.26132870e-01,-5.88579595e-01,2.93256968e-01,7.18989193e-01,-3.28200787e-01,2.51567990e-01,4.55970436e-01,3.43750209e-01,-4.73284185e-01,-6.64402008e-01,8.88116062e-01,-5.16238868e-01,8.32590163e-02,-7.39666522e-01,-5.28691471e-01,-1.06099695e-01,-2.88502961e-01,3.25202256e-01,-4.44923669e-01,-4.64818180e-02,-2.26754293e-01,1.58219389e-03,3.50656629e-01,1.53646916e-01,7.38909066e-01,-3.06067020e-01,-4.89569992e-01,-2.72598445e-01,1.11555636e-01,-2.00137496e-01,5.63456833e-01,3.58633958e-02,-4.08443600e-01,1.11624211e-01,-1.09357071e+00,2.68373251e-01,4.25667614e-01,-1.21547735e+00,-4.18594807e-01,2.22510502e-01,-6.65741205e-01,-2.04312190e-01,-1.42269731e+00,6.50493652e-02,-4.06825781e-01,1.84083104e-01,-3.88906747e-01,-3.54686528e-01,1.35320961e-01,-8.34853709e-01,-6.87470555e-01,1.29885510e-01,-8.05639604e-04,1.29978463e-01,1.16984165e+00,-7.11571991e-01,-7.48421028e-02,5.73590577e-01,-4.73979503e-01,-4.27923501e-01,2.00630322e-01,-7.70654559e-01,3.99076134e-01,-1.43369526e-01,1.56565264e-01,1.91662714e-01,2.36930266e-01,-3.23362768e-01,4.01985168e-01,-4.47708517e-01,-1.55849159e-01,2.86665797e-01,9.94156152e-02,3.63992989e-01,-4.35029149e-01,-9.54179019e-02,-3.10754657e-01,-4.91348326e-01,-5.91412663e-01,-2.92592525e-01,-6.06236100e-01,-1.78955823e-01,1.33794039e-01,-5.63606739e-01,-1.28272474e-01,-4.07560080e-01,1.69357538e-01,3.26596767e-01,-7.63401210e-01,4.06888813e-01,1.64287210e-01,-1.06916659e-01,-6.73972130e-01,3.37773681e-01,-4.99458998e-01,-2.88853884e-01,-2.49814570e-01,3.64730924e-01,-2.72398531e-01,-5.76920867e-01,-2.75261253e-01,1.85070634e-01,-6.65481389e-01,-3.85854036e-01,-4.10562336e-01,-3.43167558e-02,2.05419958e-01,-1.41953385e+00,-1.92652047e-01,-4.57346320e-01],[0.5621335,0.14673905,0.15342194,-0.26426256,-1.0298324,0.22307771,-0.05334974,0.23345992,0.07595617,-0.5356544,1.0044112,0.22764811,-1.1129383,-0.4260743,0.08963569,-0.8618018,0.5495858,0.55468035,-0.04785126,0.08879741,-0.36765513,0.12575,-0.03071136,-0.16529782,0.40432778,0.35556877,-0.5671555,-0.16068678,0.28339642,1.1678712,-0.6102368,-0.3132067,-0.6099933,-0.5499711,-0.48445025,-0.85917926,-0.55860287,-0.08005786,-0.79990274,-0.58536756,-1.0839479,-0.2103703,-0.69555205,-0.00167034,0.30798054,0.04589429,-0.5939262,-0.1045453,-0.46124095,-0.72863364,0.29303333,-0.8881197,-0.97522444,-1.1902332,-0.282957,-0.9139569,-0.5052591,-0.75228256,0.09531078,-1.0206356,0.03022325,0.29308927,0.42241296,0.04956063,-0.02277976,0.27045342,0.2218728,-0.4935537,-0.33387882,0.14515372,-0.12907623,0.2767628,-1.1508882,0.17354095,-0.37590873,-0.46014854,0.49917725,-0.08597643,-0.24680136,-0.4339863,0.72594994,-0.43494478,0.15268886,0.25861263,-0.14243521,-0.11312592,0.03705163,0.01671396,-0.27011737,-0.40251273,-0.0453319,-1.2859037,-0.02781157,0.31368244,-0.42437086,-0.14343013,-1.1163517,-0.32513392,0.39441046,0.1300608,-0.76037407,-0.02267862,0.5684644,-0.17580403,-0.85240316,-0.27378827,-0.30355063,0.09202156,0.58317065,0.3393046,-0.8147657,-0.2724489,0.4905154,-0.0082665,-0.5049275,-0.08699008,-0.4289887,-0.22137772,0.23299931,-1.1105742,-0.23574623,-0.91518545,-1.4487945,-1.25785,-0.49292815,1.1285487,0.31458732,-0.00777901],[-0.22503434,0.15342188,-0.91816485,-0.24805367,-0.5310887,0.23568375,-0.89610034,-1.1307882,0.22497255,-0.20757729,-0.75495154,-0.03025821,-0.00129612,0.7124515,-0.64787734,-0.80670804,0.11346703,0.67033297,-0.2191937,-0.22408728,-0.36658892,-0.40002856,-0.22451,0.06091923,-0.2962121,-0.55380535,-0.11910338,-0.8859902,0.18787666,0.05620011,-0.4967752,0.3660246,-0.3005572,0.45313,-0.31257686,-0.38137132,-0.129889,-0.818599,0.6223991,-0.11364434,-0.42342982,-0.35377342,-0.11000095,0.62880754,-0.17367892,-0.4366638,0.34793356,-0.806502,0.2483403,-0.16688652,-0.6946775,0.17219564,-0.25200588,-0.16013853,-0.21358997,-0.1739432,-0.18255721,0.29272,0.5068801,-0.50878614,0.21420574,-0.8152648,-0.92848057,-0.7655763,0.1390783,-0.36494806,-0.10329869,0.23254044,-0.14318886,-0.2981609,0.0370086,-0.65705216,0.36555064,0.03354775,-0.05541153,0.13402402,-0.30010244,0.08992983,0.02097402,-0.716551,-0.2711412,-0.6708352,-0.12893851,0.11340597,-0.20266517,0.21888173,-0.2523021,-0.52128303,-0.585848,-0.4180318,0.14336823,-0.60313606,0.06907024,-0.15218993,-0.5021481,-0.28495333,-0.10952888,-0.2700084,-0.6717702,-0.9274524,0.22720695,-0.6809102,-0.06552647,-0.363877,0.08181915,-0.58413327,-0.41431797,0.20287293,-0.02837775,0.20859905,-0.49220276,-0.16560663,-0.4288468,-0.43996605,-0.4163253,-0.31100288,-0.21248181,-0.4706204,-0.29581988,0.61354953,0.04338972,-0.29105833,0.40259227,0.08208854,0.01288724,-0.19243012,-0.84405833,-0.2983569],[-0.07960209,0.00396148,0.3911032,0.59294313,-0.61491185,-0.1024824,-0.0935911,-0.4122501,0.6695569,-0.09049293,0.03553626,0.48651946,0.27435294,0.05973613,-0.09381676,0.20457776,0.49496454,-0.28254926,0.3374785,0.04607409,-0.32904455,0.6704758,0.13014442,-0.24474868,-0.07386461,0.42517743,-0.43527186,-0.09026828,-0.04912698,0.0365863,-0.7343449,0.9727511,0.02157176,0.48489445,-0.44265914,0.40261728,0.31732416,-0.40994436,0.98027813,0.10413146,0.2969705,-0.54703087,0.32532772,-0.4373587,0.06622133,-0.24521635,0.18164286,0.16680078,0.0997588,-0.38078207,-0.17267472,0.11009186,-0.31283915,-0.28584635,0.23598622,0.2864037,0.2034184,-0.20594254,-0.06590755,-0.38196683,0.22982168,0.05289955,-0.21311557,0.37712035,0.23059072,0.06050258,-0.12112055,0.59972566,-0.11238542,0.6995169,-0.11949555,0.33922544,-0.12052847,-0.2526758,0.24603109,-0.07351883,-0.04462074,0.06698278,-0.804227,0.39674646,0.03272714,0.12807018,-0.02443817,0.44147202,0.11555019,0.07149215,-0.5098286,-0.24387157,0.0256774,0.37524065,-0.17695995,-0.07630382,-0.17479962,-0.17867985,0.32126027,-0.48564398,-0.04407185,0.46199372,0.13626787,-0.3246257,0.4148354,0.03266562,-0.13566014,-0.11970687,-0.33067262,-0.15189023,0.16137394,-0.4534307,0.00824247,0.1906976,0.01156874,-0.23473836,-0.06258126,-0.05873229,-0.14065708,-0.07662862,-0.22963749,0.5706997,0.05603538,-0.19520238,-0.11874323,-0.88898635,0.00565725,-0.09685448,0.27974814,-0.10152099,0.38461256,-0.39851436],[9.2492662e-02,-2.2961693e-02,1.0026545e-02,2.8709745e-01,2.5089830e-01,4.2737356e-01,1.0529445e-01,1.3162237e-01,-4.1778037e-01,2.6465628e-01,-7.1538232e-02,6.7006499e-02,-1.1621162e-01,2.0577447e-01,2.1215663e-03,4.4465038e-01,-6.9032252e-01,-4.9564055e-01,-4.0965286e-01,1.9838068e-01,-1.5430878e-01,7.8638226e-02,-5.7189518e-01,-6.2069005e-01,-2.1576232e-01,-7.0742118e-01,-3.3091405e-01,2.6431119e-01,5.4112557e-02,-3.4118807e-01,-3.9769545e-01,3.8249853e-01,2.7821645e-01,-5.1433712e-01,6.5528676e-02,-2.4001175e-01,3.0522457e-01,2.5961187e-01,-1.0285815e-01,3.4912589e-01,1.2442353e-05,7.0673549e-01,-6.6709615e-02,-9.2380904e-02,3.1726030e-01,2.0537730e-01,5.5759765e-02,-2.1732229e-01,6.0797477e-01,-3.8267249e-01,5.4835600e-01,-1.8644346e-01,-7.6528770e-01,-2.1178801e-01,-4.5170474e-01,1.2228118e-02,-3.5389656e-01,5.6287456e-01,-4.6505111e-01,3.9601922e-01,-2.1668004e-02,2.5642508e-01,2.6670504e-01,-4.2294747e-01,-1.8539651e-01,5.2059483e-02,2.8468239e-01,-4.6945137e-01,2.2769207e-01,2.9370552e-01,4.0981144e-01,-9.4072267e-02,-9.1351941e-02,7.0535056e-02,-6.8046337e-01,2.5191334e-01,1.6152498e-01,-4.1834006e-01,3.4100080e-01,-1.7004815e-01,1.8746860e-01,2.6103064e-01,9.5386982e-02,-2.4587049e-01,1.4195197e-02,2.7377114e-01,5.0253294e-02,6.2624693e-01,1.3489744e-01,1.6190919e-01,1.2778027e-01,-2.9507658e-01,-7.9137361e-01,-7.7528030e-02,3.5795769e-01,7.9240851e-02,-8.2761273e-02,-7.3697239e-02,6.0219133e-01,-1.5177888e-01,-9.4749612e-01,-1.3552399e-01,1.9743013e-01,-5.3999400e-01,-4.1661203e-01,1.5058488e-01,1.7461556e-01,2.4321152e-01,1.2530202e-01,2.6680822e-02,-1.0433136e-02,-6.5136817e-03,-1.5643795e-01,-1.9856149e-01,2.9073796e-01,1.3922560e-01,8.4466599e-02,-2.6686880e-01,3.1026110e-02,2.6261628e-01,-1.5453377e-01,9.1200751e-01,1.2726322e-01,2.5582882e-02,-3.3095121e-01,-7.3453319e-01,-2.0343801e-01,3.4638026e-01],[-0.8366207,-0.56250155,0.08132833,0.21277638,-0.20213093,-0.83049476,-0.43836853,-0.32700327,-0.8189934,-0.42924017,-0.51323324,-0.64291716,-0.4703182,0.10514693,-0.13743976,0.29427844,0.23818505,-0.03693645,-0.78526646,-0.231117,0.6163443,-0.10836247,0.11286689,-0.5340233,0.10601899,-0.7140023,-0.15601361,-0.19014882,0.5558295,-0.2989391,0.27053228,-0.8425194,-0.04383105,-0.60314524,-0.11512908,-0.38481867,0.18428312,-0.50223297,-0.04148963,-0.02086302,-0.56882864,-0.33497843,-0.09607114,0.2516292,0.2898932,-0.57530546,-0.07376655,0.12226495,-0.49263036,0.3440344,-0.8037126,0.13854809,-0.521628,0.38609535,-0.37935123,-0.2638567,0.32335538,-0.8908581,0.07562307,0.07545837,-0.99174637,-0.06766021,0.33386564,-0.69280416,-0.03158987,-0.13569234,-0.46230984,-0.51336676,0.17528713,-0.436427,-0.44673482,0.09119939,-1.0353473,0.45836517,0.17912841,-0.10014884,0.11580417,-0.0477867,-0.10804801,-0.18500093,-0.23183446,-0.785507,0.14410384,-0.5854928,-0.553344,-1.3407804,-0.25999886,0.11169806,-0.7158925,0.08477619,0.12078004,-0.34643602,0.31617916,-0.49230275,-0.31899527,-0.04089049,-0.5414638,-0.26288697,-0.40059006,-0.3775983,-0.27036285,-0.1450343,-0.6675868,0.2755822,0.51069295,-0.33820835,-0.3699118,-1.0287019,0.11284862,-0.47167766,-0.39237124,-0.87343985,0.01676457,-0.10418437,0.1165572,0.43644562,-0.34154052,0.21402703,-0.52436614,0.3020495,-0.4089635,1.3544972,0.40969762,-0.06033768,-0.67642754,-0.20236278,0.1364607,0.31183368],[-0.06203077,0.02881205,0.13863023,-0.07716914,-0.32540628,-0.47736728,-0.13817558,-0.60791135,0.3138539,-0.5000466,0.09318723,0.15953696,0.19233613,-0.14854355,-0.43120146,-0.29200923,-0.3124229,-0.46661106,-0.13660035,-0.05815702,0.24857914,0.1113189,-0.16720136,-0.7482844,0.09586824,0.26907772,-0.07552661,-0.29295436,0.33377397,0.00932376,-0.54697675,-0.5405639,-0.55168253,-0.11490633,0.60864264,0.46363464,-0.35646453,-0.07381569,-0.0998321,0.13395652,-0.33581004,-0.26851064,0.1259538,-0.591666,-0.16999862,0.33694223,-0.46126866,-0.6341087,-0.08714654,0.18201011,-0.0401035,-0.11601992,-0.22175613,-0.66092676,-0.7887865,-0.00819514,0.0312535,-0.4449532,-0.15521519,-0.16001019,-0.53001344,0.4626561,-0.47318003,-0.5124773,-0.30585337,-0.0313861,0.15728301,-0.12485618,-0.09279107,-0.1980559,0.3982069,-0.4927366,-0.50326407,-0.23595038,0.04012159,0.12902549,-0.79717636,-0.25337115,-0.16786188,-0.4041097,-0.10131468,-0.13854662,0.33250475,-0.2807025,-0.20610806,-0.20669731,0.12426296,-0.13981286,-0.22576194,0.44006917,0.34497184,-0.1384216,-0.22845897,0.55543095,-0.5250846,0.208392,0.15014665,-0.15753962,-0.20106518,-0.4102525,0.7932171,0.20094837,0.05323583,-0.10032155,-0.26552922,0.3149005,-0.89228696,-0.60419565,-0.11239437,-0.01509882,-0.29614434,-0.37165877,-0.733138,0.42803144,-0.59765154,-0.42292362,-0.65701383,-0.62503266,0.09176002,-0.203752,0.13896547,0.05235698,0.3584863,-0.15927085,-0.3466043,-0.26974362,-0.31256956,-0.34809643],[0.08019908,0.71424985,-0.0785052,-0.29374602,1.029356,0.5832478,-0.20827028,-0.03096434,-0.09578263,-1.5211138,-0.39823395,-0.76635534,-0.6185045,-0.6980858,-0.6311771,-0.1279218,0.1344819,-0.08183102,-0.27697492,0.01120095,0.39282742,-0.01708956,-0.44258964,-0.07700453,0.6918811,-0.74537337,0.1976167,0.06175052,-0.29290968,-0.3356869,-0.07782583,-1.3010687,-0.402172,0.19778472,-0.500421,-1.2276864,-0.35199475,-1.064322,-0.28166386,0.31349963,-0.08563753,-0.08434043,0.04163383,-0.74238,-0.24900402,0.38434097,0.31489113,0.10796742,-0.7524117,-0.66632825,-0.80848145,0.70879334,0.2027267,-0.29171264,-1.2716378,-0.4482457,-0.02588615,-0.7592761,0.37125513,-0.8010958,-0.23360977,-1.1254511,-0.711016,0.26277506,-0.592139,-1.6039325,-1.3940389,-0.91115856,-0.36763844,-0.04416304,-0.6348145,-0.08681463,-1.1746601,-0.8808872,-0.13993223,-0.87324196,-0.5600914,0.35547337,-0.04609672,-0.670796,-0.2572413,-0.06591382,-0.9388334,-0.7440576,-1.3269577,-0.55339545,-0.707271,-0.13286687,0.31243902,0.59513396,-0.6371961,-0.00269046,0.38145024,-0.49175036,-0.783141,-0.8220457,0.1622294,-0.5866749,-0.22859614,-0.02522851,-0.559691,-0.65110594,-0.3912378,-0.46968257,-1.0409932,-1.0922023,0.5883134,-0.48596188,-0.82995325,0.277939,-0.68268865,-0.34855244,0.24743587,0.0678792,0.33499742,-0.2918251,-0.331899,-0.78949374,-1.2877352,0.18235652,0.27012062,-1.0504174,-0.54045254,-0.31263635,-0.4150972,-0.11629542,-1.3876795,0.676149],[0.5831352,-0.4616696,0.36321902,0.20220326,-0.56023306,-0.02231506,0.27660817,0.17642842,0.7246701,-0.19484127,-0.14046316,0.30015826,0.24478509,-0.04781424,0.73788303,0.33419096,-0.3028031,0.33407575,-0.25132588,-0.25873935,-0.20962566,-0.2248894,-0.01482241,-0.30059317,-0.01593413,0.22410727,-0.4005757,0.1183916,0.15564339,0.15756713,-0.11233896,0.2313962,0.40240788,0.1525022,0.4337353,0.11555704,0.41597533,0.27907127,-0.74557763,-0.32837346,0.54641443,-0.063114,0.09009895,0.26274723,-0.00603488,-0.24488422,-0.49638924,-0.2590593,0.06271035,0.10537317,-0.5439962,-0.19611304,-0.32218432,-0.26609397,-0.6291351,-0.18535872,-0.396385,0.21396376,0.43185642,0.71418494,-0.5950133,-0.26666933,0.36523026,-0.27393624,-0.3352868,0.61498356,0.38102123,-0.330911,-0.20431727,0.5470875,0.42950752,0.03447636,-0.09420142,0.02255039,-0.18481867,0.18655008,0.5091956,-0.05272966,0.11594225,0.06623846,0.14521028,-0.43550515,0.4350057,-0.20843877,-0.00278477,-0.00854886,0.0176285,0.3641661,0.33863637,0.05147851,0.37274113,-0.73307765,0.06747638,0.6939683,-0.12776126,0.37939575,0.05579982,-0.1096327,0.40931687,-0.00403103,-0.12101179,-0.16461444,0.50347805,0.46814013,0.09178892,0.18987553,0.02618945,0.05923772,-0.22837692,-0.32624844,-0.10499216,0.20710722,-0.29126298,-0.17514433,-0.31864887,-0.13594176,0.04149693,0.09727886,0.61870384,-0.09268164,-0.25804597,-0.2229673,-0.29590276,0.05428807,-0.5246795,-0.02748715,0.03671642,0.5044557],[0.40912363,-0.08947466,-0.3082116,-0.2891939,-0.40099904,-0.04957752,-0.18145722,0.14878416,-0.10602759,-0.4141725,0.730742,-0.3818357,0.08143098,-0.45699784,-1.0096141,0.03159408,0.04191098,0.03874015,-0.2942842,0.02966367,-0.326223,0.35330716,1.6286386,-1.0753653,0.3766598,-0.46886942,-1.0708869,-0.32461578,1.4196329,0.71858066,-0.41641167,-0.34152862,0.26222003,-0.57016265,-0.95008093,-0.09364289,-0.9670905,0.0265117,-0.14655586,-0.46585914,-0.4800417,-0.10300117,-0.70951664,0.14442419,-0.34436932,-0.70392895,-0.38937494,0.08201954,0.09549333,-0.20569505,0.33804572,-1.4238948,0.5896329,0.01480092,0.3585973,0.22780047,-0.88515663,0.28531957,0.4041166,-0.41115895,-1.4750396,0.6804006,0.2165774,-0.5050077,-0.777694,0.8309179,-0.57239395,-0.23190859,-0.1980433,-0.9734471,-0.8290147,0.13922174,-1.3160533,0.3359031,0.12441467,-1.0142292,0.3337531,-0.21119806,-0.82967323,-0.10999109,0.31468034,-0.5374066,0.33005717,0.01269974,0.5626468,-0.5146712,0.05747408,-0.5503751,-0.5655312,-0.36735693,0.2617951,-0.3490028,0.14493509,0.33482888,0.05863683,-0.28859004,-0.2929015,-0.22963372,0.00542778,-0.10448911,-0.6185944,-0.60418475,-0.9193854,-0.10317964,0.08375831,0.00337405,-0.09613411,-0.30006573,0.1162722,-0.29044464,-0.47437996,-0.19226255,0.29033056,0.18680254,0.14216065,-0.65916985,-0.79203916,-0.27195713,0.29711807,-1.1300961,-0.8444724,-0.09252729,-0.6003839,-0.5798662,0.33518893,0.7944158,-0.01075105,0.09605318],[-0.28662726,0.13433677,0.3046515,-0.27922687,0.10097197,-0.31591853,0.26919314,0.12797065,-0.3885261,-0.97914314,-0.13389933,0.6450222,0.41648453,0.0447359,0.2777286,-0.30611902,0.6684625,0.04741826,-0.48971736,-0.05982216,-0.6789846,-0.33306405,0.08518789,-0.6548893,-1.022694,-0.2656245,0.05263553,-0.6827556,-0.20629111,-0.4564064,-0.3027984,-0.04107606,0.25250816,0.13571998,-0.8470147,0.6540552,-0.74272436,0.00864361,-0.1533834,-0.15110786,-0.14189592,-0.47936723,-0.99522144,-0.3690764,-0.5836196,-0.61146516,0.10140634,0.13932094,-0.57450897,0.67085737,-0.4875014,-0.65118945,-0.04910833,0.5132369,0.2657633,-0.26681033,-1.0711896,0.8748518,-0.71637446,-0.651167,0.08758771,0.02528856,0.25646275,0.07051112,0.32832858,-0.32518914,0.08572814,-0.24382673,-0.41059864,-0.19760834,0.08231334,-0.34824106,-0.41510218,-0.7015119,-0.32730424,-0.4087939,0.42995006,-0.22550991,-0.25921658,0.10437085,-0.02541039,-0.6838573,0.57591844,0.41905278,-0.04311431,-0.140604,-0.50303304,0.03799433,-0.0236185,-0.31839275,-0.7650575,0.3168803,0.2619544,0.06501229,0.24812064,-0.548393,0.04786281,0.08219296,0.4545837,0.15963633,-0.17688179,0.0289515,0.17461857,-0.3412471,0.15964349,0.2562121,-0.15964632,-0.4749762,-0.17443205,-0.05899472,-0.43105185,-0.0398275,-0.06460179,-0.23502347,0.12500614,-0.5477964,-0.7817172,-0.24216871,-0.5434607,-0.32138082,-0.3811868,-0.40752667,0.2703449,-0.07940256,0.06726237,-0.16787717,-0.3846202,-0.44295537],[-0.19528683,-0.16800147,-0.59668916,-0.3507281,-0.29758903,0.2742731,-0.16727947,-0.2276319,-0.04034846,-0.05462978,-0.33158863,-0.33363953,0.3312103,-0.15120047,0.3098854,-0.5061757,-0.31207916,0.7170041,-0.06319019,-0.00348831,0.6737282,-0.11246775,-1.1108369,-0.5445488,-0.27293178,-0.20366612,-0.09875029,0.68115777,0.8151489,-0.02729036,-0.6796099,-0.8049288,0.4097274,0.02355785,-0.29922098,-0.9116004,-0.0904353,-0.13898525,-0.42209587,-1.2959073,0.5752269,-0.21106829,-0.39791307,-0.59876406,-1.0473669,-0.35827553,0.14115936,-0.5128376,-0.02406636,0.90312797,-0.45622244,0.45749703,-0.4365588,-0.5348583,-0.39802352,-0.747092,0.23386523,-0.43870196,0.17460653,1.4556489,0.04559541,0.26100078,-1.1628195,-0.23487316,-0.21735732,-0.07227892,0.47159493,-0.6132667,0.07937201,0.27313676,-0.73970217,-0.62204397,-0.81503147,-0.09090001,-0.34186417,-0.28166953,0.39554957,-0.16359863,-0.16488832,0.18529585,-0.44794562,0.09521905,0.07312218,-0.23744519,-0.09805452,-0.20638315,1.0486474,-0.13560784,-0.10150215,-1.0890986,-1.0270069,-0.92116886,-0.5489682,-0.8302444,0.33301994,0.50478876,0.07206266,-0.5973567,-0.43268824,0.7283365,-0.24488276,-0.6062469,-0.00648254,0.21282272,-0.00768545,0.15802911,-0.1424756,0.2751008,-0.39171273,-0.8704075,-0.6342068,-0.8720366,-0.11273086,0.40067485,-0.31746915,0.9009974,0.5760921,-0.14649563,-0.24839392,-0.6807863,0.0803171,-0.98164386,0.03372036,-0.11535018,0.06096733,-0.08305628,-0.5312133,0.05986212],[0.16926394,-1.2873175,0.28583857,-0.44724914,0.29914892,-0.17780004,-0.6777295,0.14165395,0.1529711,-0.14595443,0.27049837,0.16256526,-0.44936198,-0.9665736,-0.05481869,-0.32877874,-0.3393945,-0.55279815,0.24962777,-0.33868408,0.28636256,-0.22509071,-0.26948518,0.16340248,-0.3049144,-0.52976555,-0.06867115,-0.39073256,0.1698783,-0.05447083,-0.15771899,0.5160071,0.44069374,-0.185232,-0.5275846,1.5985241,0.12839228,-0.96472,-0.27630132,0.38610208,-0.92827827,-1.2922752,-0.55535674,-0.4570676,0.44087857,0.04468914,0.37662902,-0.11556545,-0.9158323,-0.42961553,-0.31313765,0.23797597,0.13301532,-0.5709863,-0.24999195,-0.22748256,-0.8089651,-0.12466404,-0.7466925,0.0542933,-0.24850087,0.09199233,-0.10687158,-0.4379963,-0.5646338,-0.13854314,0.19505244,-0.19701022,0.24088883,-0.14531334,-0.46759814,-0.28457886,-0.8819365,-0.41640723,0.09003098,-0.2230917,-0.6281128,0.01570398,-0.0840736,-0.36130843,-0.1764249,-0.7284903,0.2835636,0.35588443,0.8567675,-0.23289561,-0.30282807,0.15320598,-0.00889384,-0.56482875,-0.6463431,-0.5734241,-0.01157309,-0.38849476,-1.0580796,-0.6647786,-0.59598947,-0.48839098,-0.13308455,0.47602037,-0.68844914,0.16444725,-0.9002538,0.33051327,-0.19859491,-0.01958188,1.0467794,-0.9085354,-0.43590754,-0.07634014,-0.78637695,-0.41873178,0.1371097,-0.26556826,-0.62789613,-0.24969302,-0.98777115,-0.8958899,-0.7402717,-0.8477437,-0.3465786,-1.2368803,-0.7096721,0.08103434,0.5778427,0.19054115,0.49844423,-0.49894306],[0.13125357,-0.99613625,-0.45361087,-0.8315938,-0.28053877,0.55329126,-0.6183568,0.22452419,0.43863168,0.5248236,-0.34284636,-0.20408647,-0.399765,0.3198985,-0.39437154,-0.065065,0.3194697,0.1619023,-0.96010876,-0.05040747,-0.3826518,-0.8073554,-0.922477,0.510243,-0.2860677,0.6670477,-0.57380897,-0.41241542,-0.86972576,0.7898075,-0.4049255,0.1821269,0.6902868,-0.7958165,0.4332384,-0.39390916,-0.29601613,-0.4194619,-0.8531977,-0.17680933,0.23164654,-0.5036538,-0.19338925,-0.7764607,-0.17138259,-0.23217744,-0.30957514,-0.95202816,-1.1697,-1.5420102,-1.10402,0.48705575,-0.30102813,-1.324116,-0.22511826,-0.02686639,-0.17655082,-0.26149568,0.01391843,0.3078837,-0.17419624,-0.37853664,-0.7992473,0.24050184,0.17770918,-0.527324,-0.2396702,0.34092692,-0.30528957,0.19326656,-0.61291045,-0.5408839,-0.5473389,-0.09439195,0.09200087,-0.5594174,-0.5310832,-0.26158646,-0.3771976,0.4707676,0.2837675,-0.6011815,-0.4997732,0.076409,-0.50818324,-0.35804582,-0.33934084,-0.58859706,-1.2709785,-0.8194775,0.55100137,0.07131933,-1.230762,-0.7239461,-0.3793768,-0.33670026,-0.17658214,-0.03492478,-0.6228577,-0.12833568,-0.4213074,0.41158244,-0.64858043,0.36419728,0.39674804,-0.87939334,-1.4697406,-0.06717666,-0.21005745,0.17838861,-0.01166187,-0.41827172,-0.22337414,-0.1697767,-0.82387877,0.03865738,-0.24580012,0.94594294,-0.67977566,0.339039,-0.6874872,-0.2391071,-0.6704711,0.21266903,-0.54579824,0.04650908,0.4535242,-0.20137852],[-0.9396196,0.3838976,0.20148598,-0.15270184,-0.26096284,-0.37888724,-0.04534556,-0.6993323,-0.0619064,0.18981802,0.10291669,-0.2683043,-0.12495843,0.02413745,-0.12982038,0.55863684,0.01528844,-0.05723161,-0.2788445,-0.747129,-0.09037004,-0.0260445,-0.37594965,-0.8321261,-0.04103931,0.12815717,-0.40596396,-0.1934446,-0.13282008,-0.03877624,-0.38552085,0.12191012,-0.6725014,-0.23332335,-0.13915507,0.33472127,-0.43472221,-0.5028682,0.5912889,0.03678751,0.3150958,-0.19210391,0.36838242,-0.54769593,0.10082866,-0.03233811,-0.11429722,-0.5257253,-0.49432194,0.4372215,-0.48641804,-0.3755924,0.15226072,-0.18441042,0.06270343,0.14618465,-0.5641019,0.31174543,-0.7357834,-0.40013602,-0.32064357,0.45349744,-0.2500101,0.13999785,-0.60067207,-0.44517276,-0.66938365,-0.03316724,-0.69147605,-0.77906424,0.3229796,-0.32361022,0.14896104,-0.35744035,0.16995615,-0.60975647,-0.51518255,-0.0755595,0.18758826,-0.8173261,-0.39280483,0.22001454,-0.24533388,-0.05375606,-0.48405266,-0.56028974,-0.37065658,-0.01245444,-0.08248301,0.28615293,0.26219118,-0.2905562,-0.074393,0.13531595,-0.57386774,0.33166504,-0.27563378,-0.13530585,-0.5255436,0.14232703,-0.07494276,0.08693741,-0.14375007,0.15275106,0.18769214,0.09335081,0.28224644,-0.05276236,0.5889587,-0.17500798,-0.80410916,-0.78279173,0.23599586,-0.22035883,-0.17880046,-0.4351164,0.08864809,-0.5752246,0.2808878,0.16516931,-0.6562592,-0.27861148,0.07336674,-0.15380931,0.20018724,-0.6313062,-0.25793576,-0.7354589],[0.0705151,0.01806501,-0.28042862,-0.79870814,-0.45132682,-0.36440623,-0.87861526,-0.0196987,0.1330858,0.11932848,0.40034673,-0.05500569,-0.28246373,-0.6035972,0.20740125,-1.2651333,-0.6587831,-0.71996206,0.0951951,0.17348975,0.89993244,-1.1470044,-1.0971282,-1.6540861,-0.49538493,0.53003365,-0.21282318,0.08452597,0.56941175,-0.6492366,0.22336812,0.5374488,-1.1024895,0.08427649,-0.9140491,-0.5347402,-0.10886039,-0.18057512,-1.0616376,-0.10418626,0.44697815,0.00360549,-0.6198117,0.4794111,-1.039639,0.10242569,-0.03214103,0.54771245,-0.65855587,1.0267756,-0.92824394,-0.9482205,-0.7438076,-0.10210195,0.14469194,-1.3975724,0.41949213,0.4210817,0.3630896,0.7482168,-0.20275652,0.24160299,-1.1924951,0.5398875,0.5357793,-0.67523134,-1.6594303,-0.7866597,0.25743312,-0.31659907,-0.47653547,-0.6767603,-0.98338574,-0.14286746,0.157097,-0.18141863,-0.2652849,-0.03310305,-0.23045123,0.19371659,-0.44542003,-0.5873295,0.02842539,-0.18530858,-0.12058072,-0.19517548,0.25726548,0.45130047,0.60886127,-1.1173253,0.7022934,-1.1663026,-0.52707446,-0.51815045,-0.46409053,0.10041922,-0.59808296,-0.72881377,-0.03653392,0.08429419,-0.10447542,-0.98734987,-0.1830425,-0.05372083,-0.689656,-0.7131505,-0.54238987,0.00937095,-0.3787828,-1.5606107,-0.39372066,-0.6317272,-0.7148427,-0.8721659,0.62509334,0.43725982,0.2631183,0.01100001,-0.6255902,-0.66466707,-0.91867536,-0.50398344,0.6854719,-0.01200965,0.11118376,-1.0387398,-0.25254592,-0.02099815],[-0.4626412,0.22125942,0.16093004,-0.05976057,-0.9676443,-0.15707967,-0.10419476,0.09938502,0.25922364,0.39726508,-0.03782594,-0.28909457,-0.29810664,0.18003413,-0.36980885,0.12853035,0.31889403,0.11315834,0.3731146,-0.2420318,-0.5255183,-0.22960071,-0.40151185,-0.3037005,-0.14046475,-0.0398413,-0.31642312,0.2467562,-0.16239506,0.26629055,0.05827497,0.21937566,-0.15012278,-0.35616082,-0.28734523,-0.30707633,0.1558904,-0.42389837,0.38627204,-0.5503152,0.34263238,0.44715443,0.26096383,0.40901363,-0.3959596,-0.24817166,-0.13305157,0.44753608,0.27291286,0.3319917,0.54556865,-0.67660034,0.17424949,-0.35392338,0.21161687,-1.0148027,-0.14827004,-0.11858635,0.2987513,-0.21910565,0.28058943,-0.09069648,-0.08242844,0.47524616,-0.52498746,-0.1833634,0.17032187,1.0788558,-0.26212963,0.03641285,0.35023296,0.9164472,-0.01397559,-0.377741,-0.6568703,-0.01286868,0.00303394,0.0548269,0.19018969,-0.26254225,0.4993322,-0.13382098,0.3445113,0.22515129,-0.1912369,-0.44216275,0.27474177,-0.00967979,-0.04656497,0.5829,0.09082631,-0.31262627,-0.19302304,0.60339147,0.4220309,-0.33604878,-0.11531086,0.08795422,-0.16410893,0.13870987,-0.10494465,-0.0363494,0.31639493,0.29744875,-0.12571266,0.02429549,-0.28869313,0.2496828,-0.29979506,-0.051117,-0.20241782,0.05921819,0.20205227,-0.55943555,0.11364631,-0.11757252,0.04870339,-0.47061232,0.5997121,0.01425719,0.5129358,-1.1477216,0.30463168,-0.3257188,0.299284,0.15557402,-0.07212813,0.4497478],[-3.83007601e-02,-4.02762115e-01,-6.20739520e-01,3.28556411e-02,-9.72296000e-02,-2.85244882e-01,3.05657417e-01,-3.55322063e-01,-5.73705062e-02,-8.85922253e-01,3.51922363e-01,-5.05557656e-02,-2.00857684e-01,-3.36128026e-01,1.05786569e-01,-4.04465467e-01,-2.77716339e-01,-5.87196469e-01,1.01007171e-01,-7.69583523e-01,-1.86968744e-01,-2.11324438e-01,4.56698518e-03,2.91012675e-01,-1.50884569e-01,6.84046566e-01,-2.63484061e-01,-3.07876855e-01,-5.74801564e-01,2.22546473e-01,2.52164394e-01,-1.62455007e-01,-2.32349455e-01,-6.57472253e-01,-6.37325794e-02,-3.45804155e-01,-3.72453660e-01,8.84585679e-02,2.53545523e-01,-3.49877536e-01,-5.00397980e-01,-3.09084505e-02,-1.53554022e-01,-5.63628554e-01,-6.65986538e-01,-2.48021167e-02,-7.59231225e-02,-3.53527308e-01,-5.91835499e-01,7.59100914e-02,-2.09534124e-01,1.58550143e-01,-4.54488218e-01,5.72034419e-01,-6.40689909e-01,-6.53902948e-01,-6.92817688e-01,8.02405179e-02,-7.69779682e-01,-4.14045721e-01,-1.14422835e-01,-4.45339113e-01,-8.29270065e-01,-3.56507361e-01,2.35568210e-01,6.32238388e-02,-7.57314563e-02,2.92313725e-01,6.04572833e-01,-1.84331939e-01,3.11226726e-01,-6.80209458e-01,2.44774893e-01,2.86603980e-02,-5.20368934e-01,4.36916761e-02,-8.07671309e-01,-5.93871586e-02,-3.30583692e-01,-8.05871367e-01,2.61876792e-01,4.53500867e-01,6.56768505e-04,-1.01803541e-01,-5.62804453e-02,-4.20105428e-01,1.44777805e-01,6.91550255e-01,-2.66514331e-01,-2.66519159e-01,-8.16482604e-02,-2.89333045e-01,-8.60625654e-02,3.63482237e-01,-4.37837780e-01,-1.09834456e+00,-3.59292775e-01,2.00705037e-01,-2.40457296e-01,-3.74607861e-01,-5.94097786e-02,-5.48875749e-01,-2.49960646e-01,-5.44859171e-01,6.88300878e-02,-3.92920077e-02,-3.10383469e-01,8.75800401e-02,-6.14080012e-01,-1.99958056e-01,-2.21666723e-01,-8.10365528e-02,3.13956559e-01,-4.83741015e-01,-1.71384230e-01,1.68496490e-01,-9.04726088e-02,3.18615556e-01,1.47404065e-02,2.66299754e-01,-8.14763550e-03,-5.75430810e-01,1.16853699e-01,5.14566302e-02,-1.80287078e-01,1.02596767e-01,-3.17646116e-01,-6.19089603e-01],[0.4659693,-0.43442068,-0.23190667,0.07656493,0.3373629,0.13347878,-0.00855935,-0.1196338,-0.0885312,-0.18399604,-0.4733811,0.1208106,0.22477113,-0.8789595,0.11851417,-0.55478865,0.113202,-0.25086156,0.32756132,0.18523039,-0.25456047,-0.34634078,0.07603073,0.44725007,0.03006241,-0.3624114,0.3386259,-0.12798937,0.21703552,-0.09659054,-0.42315704,0.1636228,-0.2391416,0.23897217,-0.09784926,0.15035015,-0.06820027,0.29552746,-0.7384457,0.70916635,0.37132567,-0.26273656,0.36445323,-0.8897989,0.1141077,0.56422347,-0.07395981,-0.05776212,-0.44393888,0.24808569,0.17252329,0.25824964,0.6301181,0.3178852,0.03749087,0.4437059,0.1582749,0.14315361,0.3118958,-0.33442092,-0.16375424,-0.37477046,-0.19252022,0.03559332,0.27259415,-0.17983556,-0.1657058,-0.8260077,0.2625996,0.37887737,0.34408405,-0.41970798,-0.08076179,-0.32354888,0.35601765,0.53870803,0.08466528,-0.31135154,-0.4049042,0.01282267,-0.28693846,0.09950662,-0.26884845,-0.5239635,0.7441382,0.15749782,-0.6153565,-0.55452543,0.02402125,-0.25562614,0.03521168,0.35663137,0.10401013,-0.15222198,0.0826806,-0.31318328,0.535375,-0.13581522,0.3577815,-0.4794712,-0.21432737,0.12098723,-0.231568,-0.13556981,0.03130803,0.83001566,0.26800328,-0.13195549,0.11632747,0.3501634,-0.17477722,-0.36541697,0.03011484,0.38165298,0.30965024,0.01534173,-0.08107807,-0.13220328,-0.2967009,0.10560444,-0.20874661,-0.4194171,-0.22327805,0.33198905,0.05416769,0.05977998,0.4204019,0.13139854],[-1.5173718e-01,1.7951870e-01,2.0160213e-01,-2.6008347e-01,-8.3555788e-01,-1.1931968e-01,-5.0740850e-01,3.6224183e-01,-8.4338129e-02,3.5750508e-01,1.6281274e-01,4.7898740e-01,3.7445432e-01,1.8903481e-02,-8.5254304e-02,-1.2532541e-01,-5.2143610e-01,-1.7116889e-01,-1.3250499e-02,5.7580777e-02,-2.5281179e-01,3.5699469e-01,2.1330047e-01,-2.7405959e-01,8.5736170e-02,-5.1211798e-01,-4.9980334e-03,7.4507928e-01,5.4402912e-01,-1.9731389e-01,2.5119114e-01,9.7162938e-03,2.3122792e-01,5.6227946e-01,3.8787591e-01,1.4345460e-01,-6.0054201e-01,3.4433392e-01,-6.2644854e-02,-2.4492006e-01,3.0230516e-01,-1.1931187e-01,-5.3470248e-01,-2.4512087e-01,2.0536518e-01,-2.6364309e-01,-8.6932808e-01,-3.2834238e-01,-3.8729835e-01,-5.8034462e-01,4.0009695e-01,3.4109908e-01,3.0805218e-01,3.2713804e-01,3.2381730e-03,-1.3115729e-01,-9.0497144e-02,-1.5085277e-01,2.0643277e-02,-4.0673148e-02,-3.3464152e-01,-2.6838113e-02,1.9116862e-01,1.1099486e-01,1.5797812e-01,3.1475580e-01,-2.2831878e-01,2.4626486e-01,-7.3350235e-03,-1.7161927e-01,-2.3559675e-01,2.6601741e-01,9.7752079e-02,-1.3384874e-01,-2.7157393e-01,-2.0405376e-01,2.7962974e-01,-2.4315834e-01,1.1222299e-01,-1.0743852e-01,4.3711320e-02,-6.5842205e-01,1.7919278e-01,2.2929958e-01,4.0661463e-01,-7.2683720e-04,-5.7079113e-01,5.0860190e-01,-1.9164975e-01,1.2805529e-01,-2.8147054e-01,2.8530534e-02,-1.3555405e-01,-1.1670818e-01,-3.0048311e-01,-5.5936743e-02,-8.2431269e-01,-4.0675512e-01,3.2038012e-01,3.4359984e-02,-6.9151908e-01,2.6000184e-01,-1.3824938e-02,1.6419424e-01,3.7518317e-01,4.8088652e-01,2.5826615e-01,-7.1572179e-01,2.4089202e-02,2.1243885e-01,1.6657536e-01,-1.6557600e-01,6.0024953e-01,3.9947346e-01,-6.0446572e-01,2.7814320e-01,-3.0287164e-01,3.7868136e-01,4.9777586e-02,9.6533686e-02,-3.2715029e-01,5.4637790e-02,3.9302114e-01,1.4799130e-01,3.3075798e-02,-4.9281159e-01,-2.0660789e-01,-5.5715615e-01]])
net.state_dict()['block2.2.bias'][:] = torch.tensor([-0.16326194,0.49530017,-0.3518653,-0.88133067,0.90829575,-0.6290863,0.1497733,0.66960543,0.03495937,0.08782956,1.4900459,0.22825731,-0.66879106,0.41050327,0.9191593,-0.06369328,-0.108637914,-0.21580109,-0.020574208,0.19848324,0.62469065,-0.37679583,-1.9715211,0.53906715,0.42854565,0.66495144,0.20535614,0.12548213,0.3641717,0.61245507,0.23001653,0.47797966,0.5733435,0.426149,-0.52967805,0.555038,0.12839627,0.7543691,0.27517313,-0.47091174,0.3511227,0.467939,-0.072134264,0.17557424,-0.06277521,0.31031132,0.3423661,0.6904867,-0.98853016,0.14519419,-0.007367987,0.4612123,-1.1006476,-0.52472097,-0.41842914,0.28138626,0.0032442035,0.122669786,0.5562609,0.9855089,-0.16579615,-0.040490225,0.25961176,-0.3031837])
net.state_dict()['block2.4.weight'][:] = torch.tensor([[-0.08489709,-1.1958141,-0.5178467,-2.9933915,-0.4427567,-1.5253137,-1.4474474,-0.5849364,-1.3418021,-0.07571221,-0.11192554,-0.06670287,-0.891679,-1.350689,-1.3674458,-0.4320505,-0.32421216,-1.4127392,-0.30967572,-1.3912382,-0.21665911,-0.87034136,-0.503563,-1.6570582,-0.23088634,-1.1871175,-0.89730495,-0.3728654,-1.0571007,-1.4470804,-1.1478277,-1.0859727,-0.8837211,-1.0248694,-0.74756294,-1.0129958,-1.3121612,-1.1067675,-0.6684024,-0.75289524,-1.3055847,-0.598941,-1.2190053,-1.1431499,-1.5131588,-1.0220056,-1.4437517,-0.20637408,-0.4048462,-1.1957362,-1.7060857,-1.5093694,-0.49333787,-1.2972542,-1.3134505,-1.3709652,-1.4613711,-1.1991599,-0.9134937,-0.8795611,-0.42202458,-1.3302828,-0.07108112,-0.38880575],[0.00838856,-1.1198628,-0.49427778,0.79694265,-1.2369769,-1.5681117,-1.2430726,-0.60574806,-1.5359452,-1.0736496,-0.16239396,-0.25008515,-1.0110636,-0.8809835,-0.9038909,-0.63947904,-0.32486948,-1.5164069,-0.31710798,-1.8809583,-0.04443249,-0.9494383,-0.75261116,-1.6649421,-0.27948913,-1.6064359,-1.0788741,-0.55180424,-1.1806059,-1.5199963,-1.7469667,-1.0983437,-0.90010095,-0.7793694,-0.4728163,-1.4422834,-0.7688146,-1.6784937,-0.6268091,-0.87803984,-1.2742102,-0.68891627,-1.1937765,-1.191806,-1.60402,-1.0034534,-1.3528203,0.01936427,-0.6701622,-1.2268628,-1.6899792,-1.594943,-0.43122673,-1.3279729,-1.4081882,-1.4194671,-1.1314577,-0.8300603,-1.0843855,-0.7953758,-0.5241956,-1.2975045,-0.13486701,-0.28897628],[-0.01824901,-0.9056637,-0.5848498,-0.42248783,-1.3063303,-1.625569,-1.1545621,-0.36819562,-1.4851855,-1.0073829,0.06863623,-0.36155525,-0.5625269,-1.4209213,-1.2618977,-0.3852086,-0.38940752,-1.4729656,-0.37823385,-1.7974949,-0.02262094,-1.0245562,-0.56218535,-1.5991321,-0.3157902,-1.5889819,-1.0428423,-0.6417933,-1.0879523,-1.5518864,-1.8230214,-0.82237184,-0.93354934,-1.1265944,-0.7376311,-1.5632724,-1.3218039,-1.6358024,-0.71837956,-0.57292265,-0.82784224,-0.5334509,-1.1783005,-0.86896384,-1.6885964,-1.0105178,-1.3271166,-0.34485388,-0.4807769,-0.4691007,-1.7177919,-1.5584627,-0.67543066,-1.3543159,-1.7270054,-1.4153712,-1.5186825,-1.3301069,-1.0308564,-0.91522515,-0.4608799,-1.3481902,-0.27299166,-0.36426494],[-0.36935404,-0.98306924,-0.45059997,-0.95365095,-1.2901118,-1.1575391,-1.587889,-0.5427913,-1.4799803,-1.0716714,0.00400152,-0.49779183,-0.67643756,-1.460975,-1.3262284,-0.5246668,-0.4502456,-1.7456279,-0.4804233,-1.8181175,-0.08719385,-0.7055168,-0.39020804,-1.5988392,-0.1337994,-1.6063712,-1.1206554,-0.5350795,-1.3261262,-1.4831805,-1.8550211,-1.1283208,-0.9550733,-1.1519693,-0.6167758,-1.4849159,-1.4210265,-1.6130637,-0.6929042,-0.78801906,-1.3936609,-0.6189148,-0.96584886,-1.245041,-1.0032173,-1.1028433,-0.94471437,-0.14905381,-0.6708527,-1.3305856,-1.1897986,-1.5980148,-0.44830596,-1.4735094,-1.8072928,-1.4777305,-1.5071507,-1.2268294,-0.45535532,-0.9262585,-0.33759722,-0.89300895,-0.2949105,-0.53293157],[0.01607641,-1.1344271,-0.5792543,0.8916024,-1.265436,-1.6475079,-1.5812677,-0.58022076,-1.0752753,-0.78421414,0.0151597,-0.23851213,-0.9153946,-1.411941,-1.3228087,-0.59054995,-0.2903868,-1.5597614,-0.46119288,-1.8288164,-0.36456493,-1.0239385,-0.4928073,-1.2040665,-0.19549546,-1.5655003,-1.0473999,-0.6803855,-1.3321985,-1.4061933,-1.7130295,-1.1374074,-0.68449545,-1.148857,-0.75822204,-1.4365188,-1.456595,-1.573535,-0.8264253,-1.0415835,-1.3879695,-0.55919725,-1.1413321,-1.2402998,-1.5645993,-0.7415535,-1.3549976,-0.13230479,-0.5590497,-1.2070146,-1.7184842,-1.5716432,-0.39307812,-0.8485828,-1.7787755,-1.4304901,-1.4076792,-1.289191,-0.9610079,-0.8961573,-0.26129723,-1.3900958,-0.36037502,-0.25830862],[-7.4750785e-04,-4.6426815e-01,-7.2886133e-01,-1.3679663e+00,-1.1515782e+00,-1.5706651e+00,-1.4823103e+00,-5.4248136e-01,-1.5884513e+00,-8.3814597e-01,-3.0799606e-01,-2.4562235e-01,-9.2707700e-01,-1.4341486e+00,-1.2581668e+00,-3.5966757e-01,-5.4926187e-01,-1.3220097e+00,-5.6152254e-01,-1.8479912e+00,-6.1540511e-02,-9.7132999e-01,-5.4235846e-01,-1.6493735e+00,-1.4560410e-01,-1.5993892e+00,-2.9845521e-01,-5.1815075e-01,-1.3971808e+00,-1.0754269e+00,-1.7091262e+00,-1.1135843e+00,-8.2650262e-01,-1.2455858e+00,-6.1454988e-01,-1.4463300e+00,-1.4123272e+00,-1.7191733e+00,-6.2041134e-01,-9.4073009e-01,-1.2241144e+00,-4.9403223e-01,-1.2159873e+00,-1.1568753e+00,-1.6479878e+00,-1.0900109e+00,-1.3796978e+00,-2.4532032e-01,-4.8901448e-01,-1.2311527e+00,-1.6756526e+00,-1.5828615e+00,-3.5752004e-01,-1.4464704e+00,-1.7380986e+00,-1.1101669e+00,-1.4181315e+00,-1.2790687e+00,-1.1439922e+00,-6.2219018e-01,-3.1624058e-01,-1.3374577e+00,-3.6613661e-01,-4.1704151e-01],[-0.01888264,-1.0723852,-0.35102174,-0.09019204,-1.1568614,-1.5321788,-1.6175326,-0.5652811,-1.5423409,-0.75182265,-0.1217804,-0.10668012,-0.9646286,-1.311396,-1.3076953,-0.49331376,-0.23275243,-0.83543736,-0.51953804,-1.8464752,-0.10172461,-0.879761,-0.51487786,-1.5305985,0.01058083,-1.5821295,-0.9393523,-0.5909001,-0.50168216,-1.3953102,-1.7410896,-1.134591,-0.8743295,0.23367257,-0.7678545,-1.4113889,-1.2690843,-1.491014,-0.4007488,-0.84624946,-1.3830997,-0.29518056,-1.217569,-1.1363231,-1.5364242,-1.0429403,-1.4638525,-0.2420041,-0.4794154,-0.939474,-1.6563603,-1.1439852,-0.52492124,-1.3079993,-1.6557817,-1.3828194,-1.3933802,-1.0579958,-1.112286,-0.83660823,-0.46079767,-1.2313087,-0.19102846,-0.361878]])
net.state_dict()['block2.4.bias'][:] = torch.tensor([-0.7688325,-0.4080679,-0.18999806,1.1288564,0.34378707,0.26467997,-0.98984414])
net.state_dict()['block3.0.weight'][:] = torch.tensor([[-6.10274151e-02,-5.43070436e-01,1.66318700e-01,-5.84997356e-01,6.05707705e-01,4.03790981e-01,-1.07800886e-01,-3.83795232e-01,-7.57863164e-01,-8.39099512e-02,1.11875169e-01,-8.50390494e-02,5.48907280e-01,-7.79258609e-01,-8.09594452e-01,-1.48748932e-02,-1.54861733e-01,2.90271252e-01,-7.60141760e-02,-7.31862366e-01,1.44798541e+00,2.72452593e-01,1.09866094e-02,-2.51389652e-01,-7.41782129e-01,-5.99998653e-01,-5.31568646e-01,6.74510524e-02,-5.14914207e-02,-9.94488657e-01,-2.78358787e-01,2.49755055e-01,-4.36081529e-01,-1.74770221e-01,5.77395618e-01,4.30175900e-01,2.23366112e-01,8.84228870e-02,-7.75877833e-01,1.07355428e+00,-4.69059169e-01,6.27430230e-02,-4.12809756e-03,6.45309567e-01,3.47923011e-01,1.20799892e-01,4.52402353e-01,-2.26955369e-01,4.25656527e-01,6.54383242e-01,-2.60426790e-01,-2.10417300e-01,-1.74694121e-01,3.56523246e-01,-6.37723565e-01,2.22487479e-01,-2.81271905e-01,2.24779606e-01,-4.45923954e-01,8.74647722e-02,-1.21837831e+00,4.14940000e-01,2.05984756e-01,-2.25054532e-01,-7.39761770e-01,7.59563223e-02,2.15473697e-01,3.63732606e-01,8.72925743e-02,4.35449094e-01,3.13148409e-01,-2.68955287e-02,-1.81903273e-01,6.74598217e-01,-9.73232388e-01,2.83960968e-01,2.98209369e-01,-6.55961782e-02,-5.05762622e-02,6.84575140e-01,3.42332453e-01,3.82593840e-01,2.70200938e-01,-2.14697152e-01,1.44268990e-01,-8.06067407e-01,1.91866875e-01,5.84886730e-01,5.32162368e-01,-1.91569433e-01,3.69919807e-01,-1.60287008e-01,-3.50493222e-01,1.00113824e-01,-1.01128471e+00,5.13190925e-02,2.08422914e-01,-5.37344635e-01,-3.18454027e-01,5.92044711e-01,5.97761214e-01,-4.30202007e-01,-1.68980792e-01,1.26926064e-01,-1.14124730e-01,2.89033771e-01,5.96804321e-02,-5.35995364e-01,-2.64602095e-01,-2.40204036e-01,2.97353417e-01,-1.29577607e-01,-5.29234827e-01,-4.61784899e-01,-2.10634902e-01,4.75268722e-01,-3.27252112e-02,-5.02736032e-01,1.53033376e-01,1.21821955e-01,-3.88456017e-01,1.94735765e-01,4.02598828e-01,5.48010945e-01,-8.99402440e-01,-7.17895210e-01,2.04480365e-01,5.73611259e-02,-2.67833143e-01,1.41952813e-01,-6.51636198e-02,-4.56278771e-01,1.03042647e-01,-1.82447582e-02,2.48315275e-01,4.22963575e-02,-3.53334397e-02,-2.77594954e-01,-2.95655429e-01,-5.16643822e-01,2.74735093e-02,3.68252128e-01,1.50822029e-01,-1.60419822e-01,-1.82327181e-01,2.99505085e-01,-6.68156147e-01,1.62565500e-01,3.86772417e-02,1.96919531e-01,5.14230669e-01,5.24162710e-01,5.83393693e-01,-3.42008471e-01,1.65679500e-01,-3.37507814e-01,-2.23383427e-01,-2.69371420e-01,9.67424631e-01,1.84207305e-01,2.81944096e-01,-6.17114425e-01,6.44984365e-01,-1.12567365e+00,-4.53857094e-01,2.31542811e-01,-4.56654370e-01,-2.30632752e-01,-1.26948237e-01,-4.85602207e-02,-3.83769278e-03,-5.33710979e-02,-2.68602580e-01,5.69973171e-01,6.03614092e-01,1.63628571e-02,-2.72904813e-01,1.10117286e-01,-2.82453209e-01,-7.69827902e-01,1.29021347e-01,-2.89832830e-01,-1.22579145e+00,6.67821094e-02,5.00921488e-01,-1.85705513e-01,2.63617069e-01,-6.70105696e-01,2.34870985e-01,-9.95688736e-01,4.88556534e-01,-2.11590871e-01,7.45297149e-02,8.25856179e-02,-5.46179175e-01,-2.77677327e-01,-1.25385635e-02,-6.53571665e-01,-2.23325580e-01,4.20287937e-01,-6.57255277e-02,-9.48700964e-01,-9.71230268e-01,3.06213051e-01,-3.23226750e-02,-3.95596594e-01,6.25982434e-02,1.33808479e-01,-1.80102494e-02,-6.30145431e-01,-4.69729230e-02,2.52060622e-01,2.27359235e-01,-4.59816277e-01,3.50504071e-01,-1.67072877e-01,1.87880695e-01,-1.31326541e-01,9.80085284e-02,-8.30209181e-02,-3.76220256e-01,-1.29482597e-01,-1.53535530e-01,-1.58679672e-02,-2.88865387e-01,2.04291567e-01,4.60552126e-01,7.16951792e-04,2.39647374e-01,-7.67963231e-01,3.62356544e-01,-2.64028341e-01,3.96662265e-01,-3.66840035e-01,-4.50546563e-01,-4.87052679e-01,3.70101005e-01,-9.11381125e-01,3.83709162e-01,-7.17459440e-01],[-0.864393,0.38395894,-0.51140386,0.14973351,-0.4146613,-0.14095612,0.04924775,0.29129928,0.04516208,-0.34762362,0.30797228,0.20076412,0.27592602,-0.541158,-0.15104428,-0.35954672,-0.14595717,0.71867746,0.04183457,-0.44568154,-0.42258042,0.6664013,-0.6569186,-0.30260018,-0.13069482,0.4182687,-0.36888888,-0.3268868,-0.4455447,0.59360564,0.4122479,0.69389594,0.1898454,0.14653075,0.150889,0.27501327,-0.26616976,0.28446543,0.15518971,-0.27718136,-0.07362281,-0.20663963,-0.5263791,-0.64471996,-0.4908047,-1.096696,-0.90234894,0.35086071,-0.5645773,-0.2841284,-0.11133955,0.04182091,-0.4822595,-0.22129463,-0.8464543,0.12868407,-0.25134897,-0.15400288,0.12173077,-0.10083976,0.18864007,-0.35090896,0.41500923,0.44869414,0.07170711,0.09996744,-0.51202023,-0.12747741,-0.35369462,0.39066294,0.44790733,-0.15696849,0.12576945,0.04083036,-0.3672741,0.40074363,-0.3330802,0.0978393,-0.01047466,-0.02661194,-0.32789275,-0.04973077,-0.05183916,0.5461147,0.31497148,0.669378,0.6960983,-0.1515076,0.0921676,-1.0177304,0.38637254,0.66593903,-0.37691227,-0.03291896,0.8484087,-0.30841208,0.39542186,0.18095781,-0.1436393,-0.74414504,-0.09334689,-0.24176402,-0.3952903,-0.08072132,-0.2392174,-0.61099076,0.21761386,-0.9060682,0.8917585,-0.5352514,-0.25276047,0.32045427,-0.5155462,0.42858592,-0.42383575,0.10355015,-0.02696699,0.27145177,0.42778736,-0.00410511,-0.27932364,0.6478617,-0.15192293,0.5192148,0.38247207,0.29439145,-0.05341572,0.39533374,0.4350983,0.39060068,-0.4109048,-0.64527416,0.35852516,1.023394,-0.22050734,0.31765807,-0.98619473,0.20126167,-0.57745343,0.61441207,-0.02879266,0.23220171,0.17923011,-0.7335634,-0.22809511,0.815206,0.45096588,0.5716655,0.6066255,-0.01596747,-0.4856119,-0.07241654,0.02538898,0.70435596,-0.8536798,-0.31270906,-0.31956258,-0.12302569,0.07338413,0.5239098,0.00993704,-0.395271,0.6361982,-0.536741,-1.0340244,-0.2626593,0.3835812,-0.7613232,-0.05323169,0.3947633,-0.562186,-1.5812471,0.35036945,0.15222493,0.33356708,0.22254895,0.0434839,-0.0659751,0.6803199,-0.68761414,0.13788868,0.6923875,0.5251772,0.7349473,0.13294916,-0.43570828,0.31002983,-0.6718145,0.19337302,-0.15054615,-0.4038522,-0.06187122,0.5177875,0.02913276,-0.7725947,-0.04935372,-0.40847448,0.7798938,-0.5325536,0.49858558,-0.54747313,-0.48875737,-0.09812178,-0.10292262,-0.18065313,0.3442907,-0.0644303,0.24953066,-0.27674767,-0.31273293,-0.6502051,0.41107097,-0.7290714,-0.2771499,0.12368813,-0.31988722,0.46850875,0.06897348,0.36023238,0.04648014,-0.5518305,0.60344154,-0.6933447,0.5761931,-0.29794392,0.26447985,-0.19625619,-0.46077925,-0.57482326,0.42202243,0.09289208,-0.06038449,0.16365258,-0.00384684,0.3556244,0.8214797,-0.27126914,1.0016651,0.35342005,-0.37538394],[-0.27644643,-0.24940865,-0.32546052,-0.33662024,0.18853645,-0.0543324,0.05177351,0.73297834,-0.12867929,-0.03692157,0.15626219,0.03356845,-0.47089124,0.48464566,-0.25746468,-0.08396289,0.5673295,0.3442926,-0.63367057,-0.86257887,-0.5609213,-0.18297887,-0.46399215,0.10907934,0.04593261,0.20778294,-0.4848004,-0.589013,-0.16294095,0.4130307,0.52717143,0.21095058,-0.1813532,0.03433797,0.7973794,0.07901683,-0.10173424,-0.6249215,0.0192753,-0.6509862,0.28589386,0.19960402,0.14214712,-0.47937012,0.6281468,0.62900764,-0.23568621,-0.37153214,-0.24791089,0.25420165,0.28416508,0.25655365,-0.5325537,-0.3862427,0.38751814,0.04733474,-0.57531947,0.01070308,0.13326624,0.81845194,-0.33634108,0.4934434,-1.0349015,-0.950071,-0.4374929,0.00862104,0.07533171,0.03818425,-0.8581414,-0.6775752,0.8660721,0.09101605,-0.27101693,0.09651954,0.14182936,0.30774894,-0.27704436,0.03138844,0.43003982,-0.6707644,-0.3582145,-0.12119859,-0.46379778,-1.0172403,0.02243411,-0.21618515,0.99566007,0.59902304,0.15408228,0.12812798,-0.24665315,-0.18613979,0.27072126,1.0924785,0.0653246,0.03912929,0.41343865,-0.2200925,0.83233476,-0.51555043,0.27365807,-0.8491653,-0.50922066,-0.53879297,0.01335558,0.13302647,-0.7048311,-0.129754,-0.46269304,0.21442284,-0.36285606,-0.02087863,-0.37131473,0.18073699,-0.10909544,-0.36011717,0.11219047,0.4170524,-0.16751023,0.11305114,0.19600597,0.586147,-0.08966434,-0.08658107,-0.0878059,0.15576981,-0.12801756,-0.41049314,0.06973444,-0.07538437,-0.12303188,0.09850363,-0.62800163,-0.05153547,1.2405784,1.3624967,-0.07489741,0.4643453,0.207566,0.03322392,0.04375738,0.23940133,0.1022708,-0.60054386,-0.25962818,0.14139046,0.49868235,-0.31861794,0.3114213,-0.780949,-0.15429142,-0.08371332,0.06337589,-0.29113832,-0.8570802,-0.31165308,-0.02015965,-0.1633382,-0.17878792,-0.4644113,-0.04621606,-0.06908868,-0.28431067,-0.30971533,-0.0756695,-0.25777647,-0.7750488,-0.30297148,-0.46217963,-0.19214568,-0.1277912,-0.38313362,-0.374116,-0.23053797,0.24439092,-0.878985,0.3591235,-0.48133197,-0.19065896,-0.49229106,0.6568634,-0.10906012,0.2183199,0.05247608,-0.62058586,-0.708956,-0.01211059,0.7377173,-0.21377309,0.00933219,0.18839435,-0.32421127,-0.7935013,-0.30881187,-0.17860259,-0.4254006,-0.08179311,0.11831,-0.08416563,0.3223746,0.1827178,-0.27653328,0.5765378,0.22750175,0.06205492,-1.0877746,0.6309453,0.7293201,-0.27287355,0.39708573,-0.27709442,-0.23300731,0.22349758,0.15435086,-0.3320947,0.00515465,0.15285705,-0.04229524,-0.6247789,0.2078262,0.0309582,0.34170386,-0.10146526,-0.64297915,0.02126814,0.10958239,0.79030377,1.0604687,0.08942385,0.2982186,0.46532333,-0.27324378,0.412803,0.4578869,0.08395563,0.09902618,-0.26732826,-0.45638576,0.15870479,-0.5941358],[-0.0444528,0.20146738,-0.16565242,0.07137392,0.16864038,-0.93607426,-0.1468266,-0.52838975,-0.1565718,0.28518575,0.16481663,0.583988,0.6462854,-1.0776777,0.17312099,0.07264136,0.2370704,0.18443418,0.05087986,0.41244456,0.7224767,-0.30980802,0.13674287,-0.5271663,0.10450697,0.2747116,-0.12890264,-0.18312594,0.01498775,0.628825,-0.14647186,0.24036781,-0.7100736,-0.50094837,-0.4798367,-0.22618619,-0.1229222,0.59331405,-0.07373768,1.0502722,-0.68986464,-0.24593024,0.40494418,0.81291735,0.14295654,-0.10478346,-0.12856536,0.71521515,-0.19349118,0.28637135,-0.10319055,-0.10989367,0.40205356,0.47798565,-0.25955418,-0.58003503,-0.32844844,-0.01194294,0.11538953,0.5247597,-0.7399261,0.16632672,-0.31447774,-1.1218107,0.4688991,-0.13184309,0.23877579,-0.3294835,-0.47366226,-0.30705482,0.25053796,-0.13263084,-0.32612866,0.0287724,-0.14121322,-1.050789,0.8371669,-0.06540383,0.55124414,-0.10571673,0.39170372,0.25105655,-0.21592835,0.6514475,0.3706328,0.11569551,-0.3131899,0.1509088,0.14981626,-0.12149145,-0.4869182,-1.3382381,0.00699353,0.13079427,-0.7682386,-0.25872144,0.5981577,0.0073613,-0.92750543,0.17670687,-0.2178525,0.47294486,-0.15967503,-0.8587178,0.27107897,-0.27680597,0.16323969,0.14549285,-0.38187245,-0.6714651,-0.63346136,-1.1856406,0.4899456,-0.09287616,0.6846136,-0.38849458,0.16114852,0.15823084,0.84949905,-0.31276286,0.10559876,0.05141164,0.5647903,0.15552083,-0.6504291,-0.45691246,0.757242,-0.09127416,-0.48674592,-0.5079825,-0.6706574,-1.0909522,-0.54884267,-0.29848424,-0.8080758,0.15613815,0.36265835,-0.13051468,0.23440352,0.5605472,0.7375947,0.3655166,0.49842685,-0.63616604,0.15583202,-0.3290491,0.79598427,0.33348203,0.11816501,0.5146439,-0.11064016,-0.14412415,0.21717627,-0.39087915,-0.04217213,0.25259367,-0.4032406,0.12375974,0.0111982,0.6646903,-0.31253684,-0.48030788,0.13511777,0.2876465,0.07342364,0.7851078,-0.06400718,0.9356308,-0.13361044,-0.2798349,-0.78187275,-0.4762598,-0.17999925,-0.20106332,-0.06646986,0.42406014,-0.8885331,-0.5467773,0.02199984,-1.1883234,-0.51285297,-0.4923688,0.57890165,-1.1681215,0.920373,-0.32228875,0.33227104,-0.1840026,-0.38287285,-0.08358043,-0.4502025,0.4871226,0.41366306,-0.22750884,-0.00486901,-0.5456612,0.31312948,-0.2520115,0.04776988,-0.07159893,-0.19367598,0.27140364,0.28293118,0.10643551,0.4710187,-0.28113738,0.31474385,0.27623367,0.29401514,-0.4978554,-0.09887688,-0.38030785,0.21637551,-0.51118034,-0.06432193,-0.01587335,0.45354325,-0.42348588,0.14039175,-0.3943228,-0.7033576,0.49295747,0.52100456,-0.04124224,-0.3386558,-0.24385285,0.6086001,-0.43203115,-0.23816878,-0.54902273,0.01228446,0.51897657,0.5280196,0.07366457,-0.42348677,0.10794563,0.01198491,-0.47134492,0.1088213,0.5390655],[-0.27406242,0.5365619,-0.8676144,-0.22258946,-0.44992876,0.4304376,0.84588045,-0.18177845,-0.01219652,-0.08502714,-0.5083408,0.18417354,0.39587626,-0.20268638,-0.5703196,-0.3930352,0.4927913,0.22754325,0.00683602,0.1048356,0.63428324,0.71316344,-0.69089895,-0.24754421,0.510075,-1.3238243,-0.83997387,-0.32219514,0.7444723,0.8021823,-0.23986997,-0.28796548,0.30668303,-0.30339172,-0.15196957,-0.02037543,-0.22195226,-0.10267873,0.5210409,-0.3889672,-0.4138691,-0.6659303,-0.07729019,0.12574704,0.37262,-0.27850425,-0.13525352,-0.23224264,0.24262556,-0.4222966,-0.35847753,-0.07596772,-0.93876445,-0.42845413,0.21673207,0.16315937,0.15142147,-0.25812846,0.01247447,0.09456375,-0.6532798,-0.02059913,-0.7736805,0.2263773,-0.4234876,0.12500462,-0.0216073,0.7770258,0.7117028,0.52842295,-0.09332182,-0.4461057,0.7209975,0.03498562,0.44945562,-0.38444534,0.8038352,-0.18824606,0.46365166,0.12564456,-0.14548635,-0.5568563,0.12709211,0.01357972,0.17578213,0.52380556,0.00543321,-0.98130584,-0.07390419,0.1231211,0.11555962,0.41987157,0.5181972,0.07466697,0.3436325,-0.08441769,0.23891923,-0.29440746,0.58208364,0.10151721,-0.5875178,0.9676666,-0.10746855,-0.37245598,0.3816999,0.51138455,0.10815696,-0.05426817,-0.8472732,0.67810196,-0.15716508,-0.194015,-1.3702447,0.51565814,-0.17352885,-0.76123405,-0.24089366,-0.42429382,0.45692858,-1.5146629,-0.34879044,-0.10680474,0.09494314,-0.77777594,-0.8110208,0.14065811,0.13490275,-0.4109247,-0.07887551,-0.3283102,-0.31222704,-0.47361067,0.06874172,-0.14903483,-0.33322382,-0.23111652,0.28752095,-0.10172603,-0.39339378,-0.63606036,-0.18858404,0.14329301,0.38761118,0.14252438,-0.8547393,-0.07486974,-0.5741965,-0.99182665,0.31961006,-0.11800285,-0.0936657,-0.08378837,0.38099635,1.0727087,0.43899572,-0.23324573,-0.07003252,-0.14088474,-0.03521905,-0.15001556,-0.04942361,-0.2315638,0.29667458,-0.7124525,-0.44246763,0.26730314,-0.2733596,-0.16321953,0.18109263,-0.3524478,0.28244558,-0.55232763,-0.7808261,0.75190294,0.16170256,0.60942274,0.24924025,-0.37926543,-0.3344838,0.04572452,-0.14638974,0.18104114,0.07166811,0.75868964,0.6864883,-0.31951097,-0.06909306,-0.33493376,0.1734897,-0.19498299,0.3716074,0.03821114,-0.08704984,0.05170956,-0.8497309,0.59223455,-0.2951481,0.02046666,-1.0052049,-0.46605593,-0.48151797,-0.55631405,0.88188756,-0.45147482,-0.12343877,-0.33106005,-0.8938311,-0.02561872,-0.93372595,0.69160956,-0.20909858,0.8920509,0.5264652,0.2114502,-0.83592135,-0.48426422,0.00642206,-0.12353964,-0.25764993,0.06420622,0.2981243,-0.12614979,-0.05541983,-0.427763,0.03096284,0.2565139,-0.5540128,0.10754599,0.32946754,-0.55654806,-0.2772667,0.14075471,-0.7489627,-0.1607241,0.00888004,0.4682567,0.21142735,-0.6779355,-0.9566341,-0.16379412],[0.45143786,-0.19520943,0.12484115,-0.5317099,-0.28414303,0.17317806,-0.3393362,-0.56281835,-0.8705526,-0.49339998,-0.09618912,0.06821861,-0.41728166,-0.5761469,-0.3742283,-0.968197,0.39760298,-0.1104144,0.02035361,0.04852853,-0.6039299,0.28791994,-0.13084315,0.4265847,0.05371792,-0.39297274,0.33776113,-0.18878102,-0.44146946,-1.3574085,0.8685826,0.25024533,-0.18508318,0.0222437,-0.23962267,0.8567409,-0.52432317,-0.7537654,0.26493266,0.6646014,0.27054393,-0.173853,0.07958446,-0.5432229,0.28466305,-0.15686734,-0.708002,0.26845995,0.08156314,0.29851788,-0.71956897,0.06170174,-0.08967969,0.34881452,0.9041015,-0.17666842,-0.6393491,0.592997,-0.04824656,-0.11514157,0.12340258,-1.1786187,-0.09023669,-0.45573747,-0.4598377,-0.52718025,-0.12904468,0.00940948,0.8738074,-0.31932396,-0.74349296,-0.33480564,0.18117584,-0.3200215,-0.29254928,-0.53639615,-0.3769897,-0.37652513,0.22625133,-0.10338497,0.722538,-0.4350714,-0.08419945,0.15881807,-1.2899371,0.08778424,0.23071463,-0.2666931,0.26657808,0.4069497,0.10821219,0.30968845,-0.4770373,-0.05575405,-0.46923715,0.15929635,-0.9852613,0.2026064,0.09223289,-0.6331418,-0.20465109,-0.6213001,-0.33203864,-0.11380427,0.21419486,-0.00139122,-0.773581,-0.530198,-1.0240024,0.5566072,0.41453356,0.08548412,0.40890282,-0.23105861,0.07409431,-0.03279416,0.00608491,0.4711064,0.4775567,-0.96554184,0.21252531,-0.22352657,0.52236587,0.11961759,0.39761233,-1.0016856,0.2705754,0.22845994,-0.4831993,0.0459663,-0.2878818,-0.64432186,0.7307,-0.56236035,-0.05201425,0.11539737,-0.30201206,-0.16200016,0.45584524,-0.7174666,0.44863614,-0.56353056,-0.8873066,0.31914133,-0.627449,-0.59528726,0.01455224,-0.30281782,-0.601237,0.24556951,0.20766252,0.1012651,0.97410434,0.0366711,-0.13437063,0.25592604,-0.33428368,-0.10407519,-0.48533818,-0.2214973,-0.11742555,0.20664407,0.9447767,-0.25491554,0.83927655,-0.07447958,0.44278044,0.28332764,-0.3997181,0.2808877,-0.10462005,-0.42191923,0.86311316,-0.04091593,-0.443984,0.3742465,-0.24163169,0.03562696,-0.19474132,-0.20412992,-0.30351868,-0.19069213,-0.66840374,0.24209076,-0.88992727,0.27803358,0.33943066,0.26264903,0.2611965,-0.2808216,0.1013753,-0.5901346,-0.33427253,-0.29041517,0.23244812,0.28850496,0.14693224,0.51080036,0.08623226,-0.13296604,-0.05355895,0.11799032,0.27463493,0.34786636,-1.1576542,0.13421607,0.4877645,-0.8643882,-0.63725615,-0.17043166,-0.13639502,0.3589794,0.406892,-0.33897752,-0.10490852,0.38338152,-0.61065525,-0.08617997,-0.15998276,0.10005521,0.06940922,-0.8000187,-0.54550856,-0.998781,-0.24032131,0.3906162,-0.02989328,0.45462132,0.68732953,-0.5788359,0.45470536,-0.18636003,-0.84989065,0.750291,1.1335897,0.70194495,0.44654423,0.01216986,0.41149086,0.63055575],[-0.29981488,-0.8162848,0.31593245,0.21616864,-0.5737966,0.085388,-0.12349109,-0.4301304,-0.17148952,0.02579413,-0.1502652,-0.15721266,0.25003356,0.9098799,-0.9393388,-0.6741011,-0.32169464,0.08531734,-0.0839377,-0.32622913,-0.5446684,0.00728535,0.32981768,-0.5553049,-0.7853717,-0.16523239,0.9472431,-0.4004133,-0.2219659,0.4711024,0.12303671,-0.98933303,0.24178836,-0.02644612,-0.77714473,-0.53101873,-0.22551356,0.71188307,0.73339814,0.37371415,0.84978676,0.6956772,0.30616164,-0.42256925,0.68765056,0.08540747,0.12512577,-0.25768608,0.04099817,0.1213225,0.4419048,0.265097,-0.33343,0.5886344,0.38245064,-0.41691568,0.43920493,0.27511162,-0.2309541,-0.41155273,-0.33527136,-0.6594272,-0.7618339,0.3120213,-0.16066016,0.39040682,-0.2320453,0.0299093,0.30170056,-1.0463216,0.04381518,0.02305244,0.35533452,0.585956,-1.1207203,0.30576846,-0.35655606,-0.58093905,0.51278234,0.21972595,0.284863,0.13591436,-0.29624712,0.04587589,-0.03226979,-0.6586493,0.03951579,-0.01355058,-0.5725889,-0.80283743,0.12297511,0.52807426,-0.43393612,0.5200097,-0.08921604,-0.5332669,-0.22433756,0.24034484,-0.2669642,-1.1051884,-0.52074707,0.67495114,0.1753295,-0.7167897,-0.648315,-0.3933982,0.6208116,-0.37697205,-1.0489426,0.4461764,0.44021246,-0.08298597,0.08771858,0.51649606,0.32060632,-0.76864773,-0.18407314,0.60444784,-0.77685714,0.07492877,-0.5055099,0.5683531,0.07679921,0.82947314,0.6458191,-0.1618322,0.15562417,-0.45188814,-0.12107609,0.47911653,0.5025132,-0.4644258,-0.35218665,-0.6495049,-0.23511134,-0.05739982,-0.05688524,0.06396867,1.1125863,-0.08846316,-0.5950286,-0.19763198,0.36472997,0.55187994,-0.05484815,0.39389542,-0.7675474,-0.14708538,0.16035676,-0.33506292,-0.294317,0.0196636,-0.20848663,-0.05976789,-0.16794689,-0.4073209,0.03029693,0.3302406,0.29441303,-0.82449925,-1.1801771,-0.0157416,-0.17773253,-0.57760483,-0.4924848,0.18299794,-0.1806673,-0.71301943,0.0583705,0.20014563,0.10294527,-0.39567593,0.08522315,-0.3978817,0.11622585,0.11947405,-0.38855246,0.04860453,0.5891858,-0.03715122,-0.26175988,0.05725741,1.0612067,1.0915147,0.03337322,0.55349356,0.24243401,0.38765708,-0.02117491,0.16350839,0.35535276,-0.32139975,-0.06597073,0.32018223,0.3102741,-0.01146438,0.32020795,-0.74413586,-0.00454853,-0.02625227,-0.55341107,-1.1241045,0.51522404,0.44591287,-0.0697998,-0.6416298,0.6863513,-0.07445725,1.2589545,-0.27344152,0.5296418,-0.2825787,-0.27976754,-0.26960456,-0.04800922,0.07130286,-0.66904587,-0.0723975,0.19121505,-0.55298436,0.4222365,-0.2726691,-0.04342798,-0.5936686,0.24751711,0.19523771,-0.3103801,0.08988141,0.7119622,0.33464658,-0.28985825,-0.12753911,-0.6436149,0.15251388,-0.5368974,0.454644,-0.679712,0.21945138,-0.53132355,-0.2261847],[-1.50970683e-01,4.95103717e-01,4.74115670e-01,6.78937197e-01,-7.79097021e-01,-6.07294068e-02,1.61732163e-03,5.19824326e-01,-1.00998771e+00,-4.35994983e-01,-1.94050521e-01,-4.16837245e-01,4.15381610e-01,-2.84297734e-01,2.25660607e-01,3.48543644e-01,-2.37612613e-02,1.62307978e-01,-6.33166656e-02,-1.17026046e-01,6.11764014e-01,-1.39481187e-01,6.80398464e-01,7.68868089e-01,-5.95173836e-01,-6.48029387e-01,-2.58038994e-02,-2.78330952e-01,-1.62173912e-01,2.67635792e-01,-3.06595057e-01,-2.51406103e-01,-2.08601102e-01,-3.24015468e-01,-2.32515484e-01,3.09795648e-01,-2.92188346e-01,1.17566213e-02,-5.70117449e-03,-2.11623758e-01,2.31493846e-01,4.72977072e-01,1.11512589e+00,-1.73414379e-01,4.21021938e-01,1.44586802e-01,-4.42940354e-01,-3.79819453e-01,-5.40669374e-02,-9.07640085e-02,1.13361189e-03,-4.05095577e-01,1.13613360e-01,4.11921352e-01,4.38286752e-01,3.13789964e-01,3.32156003e-01,-1.90215018e-02,5.68626940e-01,1.54839262e-01,-1.70400992e-01,-7.05305398e-01,6.20478988e-01,1.92984432e-01,6.25338614e-01,5.45457184e-01,-7.95103133e-01,-4.75132704e-01,-1.98513955e-01,-7.45198369e-01,-5.93870044e-01,-3.96105736e-01,-4.80040044e-01,-1.59682229e-01,-3.57246459e-01,-6.17101312e-01,-4.08370376e-01,-3.27898175e-01,-2.11155280e-01,1.42471313e-01,-7.16019124e-02,3.27951044e-01,1.80451095e-01,8.08606595e-02,6.49464130e-01,-3.32654387e-01,5.61892211e-01,5.21525979e-01,3.73841882e-01,-9.74807620e-01,-1.33522347e-01,-5.83897114e-01,-3.13202590e-02,-6.12920821e-01,-9.34845924e-01,-6.12412572e-01,2.13755578e-01,1.82379052e-01,-3.84487033e-01,1.88731831e-02,-6.07336938e-01,-2.68290669e-01,1.89395532e-01,-5.89984119e-01,1.40610203e-01,5.98406680e-02,5.75471103e-01,-2.43572727e-01,-2.10615993e-01,-5.13876140e-01,2.26516873e-02,5.16383052e-01,-4.08241570e-01,6.53458774e-01,-4.24176991e-01,4.27374542e-01,-1.53001964e-01,8.46811831e-01,-8.96741152e-02,-6.61691070e-01,1.51968628e-01,3.54237229e-01,-3.58288139e-01,7.29374647e-01,-2.78959543e-01,-1.12384498e+00,-4.72266942e-01,2.86616951e-01,2.65976101e-01,-1.38853163e-01,-2.95614362e-01,-6.25919640e-01,4.33420658e-01,4.75435138e-01,7.02202693e-02,3.47073108e-01,-5.07563531e-01,2.54864961e-01,-2.38092259e-01,3.38325411e-01,9.98449326e-02,-1.96247554e+00,3.22722405e-01,-6.36706412e-01,-4.45302516e-01,-6.12881839e-01,1.34151161e-01,-6.91271946e-02,1.65233940e-01,-5.63153625e-02,-3.30023080e-01,-8.96265626e-01,1.36943743e-01,9.04505476e-02,2.70788848e-01,1.59236073e-01,-9.91333425e-02,5.59704602e-01,-3.22613657e-01,-3.67712945e-01,-6.81043938e-02,-9.17434990e-01,8.98160711e-02,2.59476453e-01,-1.88109115e-01,7.42200837e-02,3.53517860e-01,-2.28205562e-01,2.95179695e-01,3.07103515e-01,-8.29332590e-01,-5.35402894e-02,-2.84812778e-01,1.26451433e-01,5.29695690e-01,-1.93046201e-02,-8.20645154e-01,-3.83592248e-01,-4.32006836e-01,1.49068804e-02,-2.96344578e-01,-6.48620427e-01,5.10962665e-01,-8.77880633e-01,-8.60626459e-01,-2.47133717e-01,2.85417557e-01,9.24386904e-02,3.26608151e-01,-1.39881661e-02,-1.09423733e+00,-4.38274771e-01,3.79351646e-01,-5.18999934e-01,7.02440798e-01,-2.00198188e-01,1.62036136e-01,-2.00804368e-01,-6.91535413e-01,2.22081721e-01,-6.23875596e-02,-3.29552472e-01,-3.58585594e-03,-2.57309705e-01,5.18282712e-01,2.60851175e-01,-2.64277995e-01,3.60660404e-01,-9.96929348e-01,5.80384195e-01,3.91100198e-01,-2.52427816e-01,2.50332296e-01,-1.04151785e+00,-1.08032137e-01,1.63464770e-01,-3.26707214e-01,3.13791901e-01,-1.24212131e-01,-7.15297759e-01,-6.39946163e-02,-2.06199586e-01,-1.91294804e-01,-3.95334482e-01,4.38420653e-01,-3.23463768e-01,2.04268470e-01,-7.14463219e-02,2.67082512e-01,-6.02831304e-01,3.28079820e-01,-3.51503253e-01,-5.56162596e-01,-6.01136625e-01,-2.35844925e-02,8.95113230e-01,7.76934862e-01,-3.70615393e-01,1.78807810e-01,2.63141364e-01],[-0.51659983,-0.4428704,-0.10425762,0.22199455,-0.32979506,0.14635621,-1.0048516,0.01847153,-0.15336244,0.11228418,-0.653534,-0.163618,0.16652234,-0.20799461,-0.11151717,-0.00397425,-0.44795513,-1.1089345,0.22087598,0.45961577,0.33907104,0.26838452,0.05477189,-0.79189956,-0.14655103,-0.2409792,-0.1925634,0.16456585,0.43647495,-0.64727944,-0.4940923,-0.12098984,-0.7637921,-0.27171132,-0.08114212,0.3319621,-0.35737905,0.21803387,0.7443427,0.05508336,0.08841909,-0.10576116,-0.32256907,0.3578649,0.03855979,0.17239214,0.15168232,0.00597634,-0.0954816,-0.30088335,-0.15911005,-0.02804894,-0.38506466,0.04253035,0.59797853,0.27301946,-0.05768331,0.10565763,-0.13031067,0.25155386,0.0566151,-0.2650141,0.2832637,0.20065747,0.21747206,-0.16952321,-0.9447367,0.00636209,-0.40639442,-0.49754617,0.27190456,0.15504658,0.12113778,0.58547074,0.32154202,0.25389993,-0.5423296,-0.53776205,0.2569468,0.40972123,-0.26164302,-0.46739602,-0.08143364,-0.00767874,-0.5415088,0.62028575,-0.2656067,0.25235593,-0.4367004,0.02536493,0.48461807,0.187643,0.6340601,0.31678766,-1.0827045,-0.53776544,0.8700614,0.11638881,0.26805568,-0.12870611,-0.87125915,0.40436047,-0.13154817,-0.15433998,0.73592263,-0.00466558,0.6602989,0.17086664,0.02386548,0.1386475,-0.46213582,0.7166296,0.69460595,0.3530516,0.20116527,-0.21661958,0.18642277,0.37417436,-0.41861463,0.39576766,-0.00536466,0.4135994,-0.02313396,0.39255288,-0.62318105,0.27939016,-1.0068046,-0.11237632,-0.6818315,-0.359074,-0.20625168,-0.09847365,0.03757566,-0.15109652,-0.14476691,-0.5477879,-0.2863718,0.29806548,0.38710883,-0.03094122,-0.8647663,-0.12120374,0.40554187,0.47155142,-0.21918532,0.01460307,-0.4020218,0.5206162,0.4150497,0.34148654,-0.38101983,-0.10488888,0.9034892,-0.37869883,-0.02102594,0.10578635,-0.5243889,-0.04353663,-0.22405806,-0.07722087,-0.5802075,-0.95160943,0.01301204,-0.12373433,0.28317177,-0.40212375,0.26942188,-0.14585306,-0.2406477,0.1394654,-0.29626727,0.31365535,0.32070515,0.286544,-0.26623768,-0.50936735,-0.22875158,0.10512087,-1.054148,-1.4226516,0.9302116,-0.04725719,0.39892915,-0.02476939,0.3169233,-0.19562706,-0.15182206,-0.10870386,-0.5877356,0.39443386,0.14900623,-0.20137501,-0.23078065,0.06046422,-0.1486873,-0.12991017,0.09060761,-1.0272897,-0.01318077,0.41588128,-0.45893356,-0.06162872,-1.1261001,0.03399374,0.89701104,0.02936704,0.10011242,0.90572715,0.5278034,0.15291032,0.09217305,-0.22224793,0.08634089,0.25263008,0.53296685,0.5140824,-0.4165803,-0.34353665,0.13816255,0.3503882,0.22473437,-0.64794964,-0.5154472,-0.70372087,-0.5575696,0.22808357,0.02359698,-0.33160225,-0.06426963,-0.99396163,-0.6402238,0.11593329,0.46081632,0.05721185,-0.11198074,0.61147,-0.12304812,0.25666606,0.02357916,0.16403876],[1.06940401e+00,6.46264195e-01,4.01052862e-01,3.23205180e-02,5.49113631e-01,1.09056599e-01,-4.77055460e-01,-4.41736937e-01,8.04286078e-03,-2.27746278e-01,-2.00793799e-02,-2.11119846e-01,-4.59691435e-01,4.91079539e-02,-6.43895745e-01,-6.69646382e-01,-1.47671416e-01,8.84901509e-02,-1.42446235e-01,-3.72999936e-01,-1.25861555e-01,-1.07740057e+00,1.99706808e-01,9.27075371e-02,-1.12143338e+00,1.28177807e-01,-1.04081698e-01,4.08100694e-01,4.32948470e-01,4.15575176e-01,-5.32093108e-01,4.32921737e-01,4.88851517e-01,-4.19501930e-01,-1.28423226e+00,4.71781641e-01,6.63620651e-01,3.85609716e-01,-9.47213173e-02,2.97592103e-01,-3.68352711e-01,-1.88997254e-01,-2.13803858e-01,4.75338101e-01,-1.15153778e+00,5.35818994e-01,-2.52676725e-01,2.10449435e-02,4.47005779e-01,1.79824948e-01,3.10090154e-01,-3.62791240e-01,-1.96586460e-01,-4.86409187e-01,3.76621693e-01,-7.12633207e-02,-1.98471949e-01,6.99737221e-02,1.64816007e-02,2.63704598e-01,4.44005057e-02,6.83944404e-01,-4.79254454e-01,-8.39104727e-02,-6.36237711e-02,-1.30040109e+00,-4.13062811e-01,1.27206580e-03,4.60879236e-01,-1.51895761e+00,3.58513951e-01,-8.59869003e-01,6.25875816e-02,-2.44941086e-01,2.19314665e-01,-1.15901418e-01,-7.65979409e-01,-2.29289040e-01,-2.74666995e-01,3.16258490e-01,2.88123935e-02,-3.29012275e-01,6.99792922e-01,-2.33819142e-01,-3.52520764e-01,2.27845937e-01,7.18748331e-01,-4.69285809e-02,1.85362473e-02,1.14175938e-01,8.79963636e-02,-2.34987840e-01,-3.22520226e-01,-3.62579882e-01,-3.60803217e-01,1.13438778e-01,-5.10040335e-02,-1.41136125e-01,-4.05337334e-01,1.36789754e-01,5.67301810e-01,-2.92761326e-01,-1.29976735e-01,3.19494039e-01,-7.42394090e-01,-1.17995299e-01,5.78031316e-02,-1.59737185e-01,2.97250509e-01,6.29376292e-01,-4.90828604e-01,1.15879044e-01,-5.86510658e-01,6.45497382e-01,5.71185015e-02,5.80451302e-02,2.36575365e-01,1.81619197e-01,6.26423240e-01,-1.74749240e-01,-3.17011327e-01,-4.03748304e-02,-6.36451819e-04,-1.21024780e-01,-5.91543019e-01,3.35230589e-01,-5.88957965e-01,-1.98756263e-01,-7.18543768e-01,2.68048674e-01,-6.75358891e-01,4.22837526e-01,-2.62926280e-01,-6.64967775e-01,1.67763889e-01,-2.36045003e-01,-2.12979503e-02,-2.46977195e-01,5.56450844e-01,-9.28308308e-01,3.83552670e-01,1.20259896e-01,2.61915535e-01,4.51920450e-01,2.04467058e-01,-6.80484891e-01,3.87630343e-01,1.07734524e-01,-1.00729885e-02,-3.55341345e-01,5.83365321e-01,-6.19905293e-01,-5.28202236e-01,-4.59589303e-01,-1.12399712e-01,8.56482804e-01,2.10722968e-01,2.25869194e-01,-9.48026419e-01,-2.42866784e-01,1.29539832e-01,2.98315138e-01,3.62095594e-01,-7.49660909e-01,4.83048737e-01,-7.12769479e-02,-1.06970227e+00,-4.24084097e-01,-4.76910830e-01,3.39669406e-01,-1.45142660e-01,3.95676821e-01,3.89193714e-01,-4.41110553e-03,-7.08738640e-02,4.29500937e-01,-5.32973409e-01,3.09722543e-01,8.97210836e-01,1.06244430e-01,-5.28176725e-01,5.42928040e-01,8.60540792e-02,3.53672177e-01,6.96980804e-02,5.24307549e-01,4.88654256e-01,3.46950829e-01,1.00516856e-01,-2.39310876e-01,5.50965928e-02,1.33383438e-01,-2.31104940e-01,-3.33586007e-01,-5.34362733e-01,7.90945947e-01,-5.42204261e-01,-3.18863302e-01,6.69630468e-02,-9.40452740e-02,-4.32274014e-01,4.33577210e-01,6.17709458e-01,3.50579806e-02,2.46900037e-01,-8.53053212e-01,-3.68793488e-01,7.76110739e-02,6.79853279e-03,3.07446748e-01,2.14521274e-01,-7.49912798e-01,-4.03149456e-01,-2.49738827e-01,-9.66203585e-02,-8.02677631e-01,-9.04539943e-01,4.07239646e-01,1.33577690e-01,-1.52703598e-01,-1.13448918e-01,-3.65811825e-01,5.25067925e-01,-1.65092815e-02,1.65272355e-01,-3.57288867e-01,-1.57117635e-01,-6.26500472e-02,-7.56193519e-01,2.98315406e-01,4.17165428e-01,-1.31669676e+00,-6.85931981e-01,-1.24438137e-01,3.47436741e-02,-1.06710088e+00,-6.72493517e-01,4.17967945e-01,3.76657754e-01,3.83175224e-01],[2.80581769e-02,1.25355616e-01,-1.84173778e-01,-1.13238478e+00,-3.39927644e-01,-6.71055496e-01,1.48308948e-01,-4.24312890e-01,4.63523299e-01,1.39729619e-01,-1.32474601e-01,5.12821972e-01,7.40712762e-01,6.25655353e-01,-3.21839809e-01,-1.38780415e-01,2.13951588e-01,5.29433429e-01,3.10849011e-01,3.58485609e-01,-1.08126357e-01,-3.17641467e-01,-6.71287358e-01,3.94942582e-01,-4.62825298e-02,-2.05709208e-02,2.85419285e-01,6.35980010e-01,-1.09426327e-01,4.39079739e-02,6.16174877e-01,-2.76634723e-01,-7.91764617e-01,-1.72178745e-01,-6.93018556e-01,-1.03146337e-01,4.29677695e-01,-6.33310556e-01,-8.22013378e-01,-1.01780844e+00,-6.55713260e-01,-9.06241477e-01,1.08888455e-01,-8.22887242e-01,4.17612903e-02,1.68096527e-01,-3.81678380e-02,-1.46717918e+00,-1.00200152e+00,-4.23466504e-01,-1.25833571e-01,4.52058166e-01,3.68052006e-01,-6.04923248e-01,5.57099581e-01,-1.64934814e-01,-3.62314522e-01,-3.03064704e-01,4.17010099e-01,-5.52725829e-02,-4.53010872e-02,-1.91788539e-01,5.60930550e-01,1.64947465e-01,4.54277039e-01,-2.40159661e-01,2.74937063e-01,2.05685347e-01,4.91098076e-01,-5.46021044e-01,-6.84776127e-01,-5.89587390e-01,3.20571750e-01,-9.62958951e-03,-2.15035439e-01,2.93648779e-01,1.21915579e-01,3.29800457e-01,-4.00080383e-01,4.66954291e-01,-1.53969631e-01,-3.16083618e-02,-6.91621244e-01,6.75855160e-01,-1.38156593e-01,3.78057390e-01,-5.86211085e-01,-1.50512919e-01,7.17784107e-01,-1.52121663e-01,7.54837394e-01,-2.67872274e-01,7.79867411e-01,1.06036998e-01,1.90182537e-01,4.93246317e-02,-2.42851958e-01,3.94520879e-01,-1.83401965e-02,4.02463377e-01,7.51929343e-01,-2.05482230e-01,4.84982096e-02,-9.02688354e-02,3.92297447e-01,-2.79948324e-01,-3.49820912e-01,5.85480332e-01,7.70539343e-01,5.09072602e-01,-1.51712984e-01,7.25897253e-01,-6.24837220e-01,-1.77786574e-01,3.66560400e-01,8.32213238e-02,9.04601738e-02,-3.70719552e-01,-4.61425066e-01,-3.46497834e-01,8.62920880e-02,-2.46502161e-01,4.90932435e-01,-7.97634065e-01,1.23425171e-01,4.86761667e-02,4.88305539e-02,3.15449119e-01,5.24316579e-02,-2.84876674e-01,-3.12985271e-01,-6.53247833e-01,2.64053047e-01,2.43951365e-01,-7.12535679e-01,3.21620405e-01,1.48228809e-01,8.91508255e-03,1.30463168e-01,3.70287150e-01,3.36078256e-01,-1.07344322e-01,1.95266888e-01,-9.15181935e-02,-3.10176730e-01,3.54769468e-01,-7.43309081e-01,5.82278296e-02,1.10043846e-02,-3.68182302e-01,-6.53117672e-02,-3.81730087e-02,-2.80291457e-02,1.11193672e-01,-5.06103113e-02,3.23872507e-01,1.93109691e-01,2.39789531e-01,-1.11885655e+00,-5.56875050e-01,-7.14524865e-01,-7.05688477e-01,-1.61247775e-01,6.14188135e-01,7.21873268e-02,2.84088161e-02,-6.04458153e-01,2.39157617e-01,-3.46862571e-03,-6.59630954e-01,1.01717018e-01,2.99063295e-01,3.49860966e-01,-4.52412605e-01,7.26738811e-01,-2.70484298e-01,-2.43746042e-01,1.62984341e-01,5.18538296e-01,-1.54822779e+00,5.01530588e-01,-5.54905474e-01,2.60769546e-01,-3.75779457e-02,1.47171901e-03,-1.74100831e-01,7.40373015e-01,-6.59344196e-01,-2.34698042e-01,-2.91377574e-01,-7.48847604e-01,7.63389289e-01,4.71182823e-01,-1.77479595e-01,-7.48234242e-02,-5.46342254e-01,-7.81514883e-01,-5.72925150e-01,-4.00911942e-02,6.69298530e-01,-3.54184419e-01,1.08024442e+00,-1.47319913e-01,-1.90990835e-01,5.92670381e-01,-2.20122576e-01,4.02179360e-01,4.15936410e-01,-2.67361134e-01,-2.71753401e-01,-3.42962444e-01,-3.23993951e-01,3.22081037e-02,-3.30647081e-01,3.23531963e-02,-5.77587962e-01,-2.56230198e-02,3.88332099e-01,-4.88607049e-01,-5.75921714e-01,-2.83929795e-01,2.34641120e-01,-2.90266514e-01,8.62681866e-03,-4.38054979e-01,5.74029386e-01,5.65411925e-01,-3.25874478e-01,-1.19184494e-01,-1.53401315e+00,-8.14016610e-02,3.25176328e-01,-3.26823443e-01,1.17917046e-01,-6.78706527e-01,-5.41671872e-01,-2.96299398e-01,3.60696577e-02,8.58199820e-02,-4.32730794e-01],[-1.93241715e-01,6.84813410e-02,2.77129084e-01,-2.38174781e-01,-5.72394133e-02,6.89867914e-01,-2.80268610e-01,1.81641996e-01,2.72794187e-01,3.55732888e-01,-6.19749874e-02,5.29334471e-02,-3.66598010e-01,-1.92580268e-01,3.96272764e-02,4.19033051e-01,-8.27728748e-01,1.55016750e-01,1.03163887e-02,-1.41311318e-01,-6.89528108e-01,-5.88433325e-01,3.73956144e-01,-6.30074918e-01,2.29687586e-01,-8.77084434e-01,-1.80068925e-01,-4.53363299e-01,-2.94760495e-01,-1.71167672e-01,-3.55964363e-01,1.33247087e-02,3.51371825e-01,-4.40831542e-01,-8.06884885e-01,-1.13170654e-01,1.05155997e-01,-3.08960766e-01,1.19197987e-01,1.49947539e-01,1.46857426e-01,6.57376871e-02,-2.29763910e-01,-4.87035036e-01,-4.54032838e-01,1.21399291e-01,-6.72484338e-01,3.72596346e-02,-4.73209113e-01,-3.79163384e-01,-1.11792885e-01,-2.14037627e-01,-5.86647749e-01,3.18387538e-01,3.19588110e-02,-6.18100874e-02,-4.56521779e-01,1.00558233e+00,2.00322837e-01,-2.00660780e-01,3.52731913e-01,-3.11635017e-01,-1.02729023e-01,3.40086997e-01,5.04123211e-01,3.13021511e-01,3.66936810e-02,1.42524064e-01,-3.64289463e-01,9.87484232e-02,4.50708091e-01,3.50168347e-01,1.69668347e-01,-3.87617022e-01,3.06552857e-01,-1.92802444e-01,2.35868230e-01,-2.71894544e-01,-8.23937654e-01,-5.98496139e-01,-2.34849691e-01,5.58930039e-01,1.53017431e-01,7.82788873e-01,-2.25073949e-01,-2.95958042e-01,7.44965792e-01,1.97352365e-01,2.15881363e-01,-2.75379032e-01,-1.38288975e-01,9.83031914e-02,2.36486971e-01,-9.57178026e-02,5.88626564e-01,6.19466782e-01,-6.14774048e-01,-7.62954593e-01,5.77282786e-01,-5.37343860e-01,-6.43544495e-02,-3.36372226e-01,9.25512053e-04,-6.10459924e-01,-3.87152195e-01,-3.85449260e-01,-4.67020005e-01,8.91375482e-01,-9.84223187e-01,-5.19183517e-01,5.95077574e-01,-2.54030704e-01,-2.90612906e-01,-2.70305127e-01,3.05108100e-01,-7.55874097e-01,-1.04145682e+00,-9.52907503e-02,1.23903155e-01,3.38660926e-02,-4.03763764e-02,-6.59209430e-01,-8.96889389e-01,6.87003881e-02,1.17983393e-01,2.46454999e-01,3.30268182e-02,2.63732523e-01,6.62381709e-01,-2.17873752e-01,2.28767116e-02,-2.80401498e-01,-1.35355687e+00,2.88190812e-01,-1.06490850e+00,5.55155650e-02,7.65021622e-01,-9.45289284e-02,4.73202080e-01,-7.15096533e-01,1.55937403e-01,-2.40490921e-02,1.87622443e-01,-1.15731823e+00,4.93777543e-03,-4.32177901e-01,4.52100217e-01,-8.46321762e-01,-1.41218603e-02,1.34042159e-01,1.23186871e-01,3.24370652e-01,-4.99127418e-01,6.48006722e-02,-2.40570847e-02,-4.85547513e-01,-1.11651467e-03,4.30860996e-01,-1.52050152e-01,1.03780262e-01,6.22800887e-01,4.84599657e-02,-5.50286621e-02,-4.21329767e-01,5.70584834e-01,1.45896658e-01,-1.86104298e-01,-4.27546889e-01,6.03285730e-02,3.39536577e-01,4.92963552e-01,-3.12123299e-01,1.28541514e-01,-1.20568283e-01,3.19283813e-01,-5.22876143e-01,5.16905487e-01,6.37366772e-01,-4.44224000e-01,-5.36913574e-01,-7.51758814e-01,-6.61112189e-01,2.01929942e-01,-1.83346495e-01,6.48946345e-01,-1.90108359e-01,-3.00070196e-01,-1.57976933e-02,2.16793120e-01,-1.90645620e-01,3.27017248e-01,-4.19181213e-02,1.10971503e-01,3.13512772e-01,-4.70832288e-01,-4.32662964e-01,-2.77222544e-01,-4.57326442e-01,-2.52313942e-01,5.00328004e-01,4.95054759e-03,2.27576122e-01,6.21311069e-01,2.56108254e-01,-7.39333749e-01,-4.74233747e-01,-8.59997451e-01,1.29711954e-02,-3.54325831e-01,5.11234701e-01,3.83188516e-01,1.06249966e-01,7.78481215e-02,1.66833743e-01,-6.34681761e-01,-1.43356726e-01,-5.62570453e-01,4.12852049e-01,-1.83725245e-02,4.18081880e-01,6.68165684e-02,1.51474908e-01,1.20576195e-01,-6.55211329e-01,5.66350341e-01,-5.53339958e-01,3.38527381e-01,3.59160095e-01,-2.80116230e-01,-2.60975391e-01,-2.93884519e-02,-8.37704167e-02,2.94792503e-01,-1.83142409e-01,1.02886654e-01,-6.28564432e-02,1.06291676e+00,-9.78380978e-01,-3.19955945e-01,2.84737498e-01],[-1.87838692e-02,2.27547482e-01,-6.12899840e-01,2.26038128e-01,-7.20300078e-01,-4.51782703e-01,-1.04863107e+00,2.93715894e-01,1.12687200e-01,-2.52346575e-01,4.32936251e-01,-3.45942944e-01,4.09300625e-03,3.59723777e-01,1.68830808e-02,2.66778439e-01,7.50109971e-01,4.40560967e-01,4.09562498e-01,-3.13976735e-01,-5.53887010e-01,-7.49994516e-02,-4.32641894e-01,-3.65700632e-01,-2.35428497e-01,3.21791917e-01,4.92556483e-01,-5.70186317e-01,6.51236892e-01,1.02092870e-01,-6.52259514e-02,1.90211192e-01,-8.90788063e-02,-4.92912710e-01,-6.81852028e-02,-3.25253129e-01,-6.40111268e-01,-8.67374465e-02,6.26826525e-01,-3.02672148e-01,3.06249022e-01,-8.19397122e-02,1.71999842e-01,-4.60711241e-01,1.10472679e-01,-3.38104159e-01,-8.03819060e-01,5.88487804e-01,-1.31118461e-01,-4.17473353e-02,-1.99139282e-01,2.84236014e-01,-2.88752049e-01,5.05003259e-02,7.27091610e-01,4.38378632e-01,6.64637029e-01,8.65167499e-01,-6.34333551e-01,-3.49151909e-01,5.94378412e-01,-3.76484215e-01,2.80576587e-01,9.05535221e-02,4.41684484e-01,-5.84295690e-01,-9.32501480e-02,-8.54089618e-01,-4.69182968e-01,-8.21432650e-01,-4.40147638e-01,2.07560569e-01,2.14953527e-01,-4.09253150e-01,2.58347392e-02,1.02579877e-01,8.32783997e-01,-8.33069347e-03,-2.11787879e-01,-4.09435749e-01,-2.70163920e-02,6.70796335e-01,3.90983343e-01,3.53988230e-01,-7.90819049e-01,-1.80672124e-01,-2.76356280e-01,-4.71245855e-01,-5.66961467e-01,-1.17197251e+00,-1.42986074e-01,5.24564087e-01,-4.66126263e-01,-6.63334787e-01,7.76048243e-01,-2.07292825e-01,-1.67641826e-02,-8.53059053e-01,4.79456484e-01,3.27153534e-01,1.17059655e-01,3.55229378e-01,-6.49529815e-01,-3.92494619e-01,3.78535748e-01,9.07675475e-02,-5.40307462e-02,-3.61099429e-02,1.19496369e+00,2.18869537e-01,-2.26092190e-01,-1.19899064e-01,1.27269506e-01,-4.84631777e-01,-8.88064411e-03,1.95295867e-02,-7.36800283e-02,-3.29867154e-02,-6.15976930e-01,4.26920354e-01,-1.58983558e-01,-3.68098199e-01,-4.96022761e-01,7.26223707e-01,1.47042591e-02,-3.07184964e-01,1.04897149e-01,1.02959737e-01,-1.37147978e-01,9.55971181e-02,-5.89794397e-01,-6.88675046e-01,-3.92319292e-01,-1.28576979e-02,-2.67086506e-01,-5.47831416e-01,3.02458018e-01,-1.80953503e-01,5.03692389e-01,-5.14992289e-02,-1.17093995e-01,9.69103158e-01,3.45659405e-01,1.19899496e-01,-3.88093084e-01,-1.16445467e-01,5.69673657e-01,3.31761897e-01,-3.84897321e-01,-6.25750244e-01,-2.90829130e-02,-1.70689240e-01,4.94334221e-01,2.71083504e-01,-4.60843146e-01,5.90286076e-01,6.26623094e-01,1.22436590e-01,-8.16235617e-02,-8.41724038e-01,-2.89514095e-01,-3.84484082e-01,3.89178038e-01,-5.26914001e-01,1.57421008e-01,-3.44397843e-01,-2.25172877e-01,-1.01842038e-01,-2.55976528e-01,-1.00074254e-01,-3.36992770e-01,-7.79197991e-01,-3.71461771e-02,2.69625157e-01,6.52613997e-01,3.72765819e-04,-1.20585755e-01,-8.02972689e-02,3.87166888e-02,-3.61304253e-01,-1.19061254e-01,5.76466359e-02,-1.02254212e+00,-4.32364345e-02,-1.22600816e-01,8.61586537e-03,4.79643971e-01,-2.20311940e-01,3.01344041e-02,3.65329921e-01,-5.03276289e-01,-6.68079913e-01,2.57262379e-01,2.29513094e-01,2.11182926e-02,-2.25990295e-01,-2.77032375e-01,2.45269045e-01,4.79106963e-01,-5.53025343e-02,-2.37584025e-01,-9.44977105e-01,-4.62212823e-02,1.18464518e+00,1.42935961e-01,1.63625866e-01,7.13312924e-01,-3.09092641e-01,-1.70273691e-01,2.39196181e-01,-6.04097605e-01,-3.28169852e-01,-2.44366482e-01,8.31833541e-01,-5.69464922e-01,-5.72439134e-01,7.34322190e-01,-2.40772158e-01,-8.52786481e-01,7.58195698e-01,1.25100613e-01,-1.21981323e+00,-4.28752184e-01,1.10813297e-01,-1.16105878e+00,-5.88305116e-01,-8.76450658e-01,5.87858975e-01,-1.23873681e-01,-1.61057748e-02,6.36418998e-01,-2.84602642e-01,1.80927306e-01,1.30938947e-01,3.07909131e-01,2.09455177e-01,3.74508679e-01,3.37123275e-01,3.21581960e-01,-5.44299781e-01],[-0.34532452,-0.4113313,0.68788326,-0.08766967,-0.98198676,-0.271169,0.32209936,-0.1356806,-0.00970307,-0.07406367,-0.01118104,-0.2210109,-0.45170873,0.18664037,-0.53960854,0.2992898,0.05708949,-0.54979604,0.09727108,0.4631363,0.6955997,-0.29320788,-0.18491617,0.53445905,-1.108127,0.10925157,-0.33891773,0.1951612,0.18545854,-0.23085286,-0.5609812,0.12105832,0.52654326,0.11145834,-0.1511499,-0.28011492,-0.20799665,-0.06978414,-0.0134763,-0.18144673,0.26727676,0.8057494,0.00479235,0.30956644,0.14984463,-0.38324314,0.12010156,-0.4464384,-0.13481212,0.23215593,0.3085043,0.3076488,-0.5537138,0.73680496,0.06025023,-0.01919737,-0.13163862,0.48219392,-0.24713589,0.30140427,-0.9342357,0.7081116,0.4186576,0.26833418,0.21270606,0.22298916,0.74724746,-0.48879853,0.3262393,0.09337826,0.23563303,0.1624855,-0.39510965,-0.104405,-0.21757586,-0.6078703,0.01907744,-0.33284956,0.7243923,0.03029838,0.81956035,-0.9279572,0.2359728,0.14101076,-0.8119722,0.06493254,0.40832713,0.48013574,-0.5624946,0.17517035,-0.16426519,0.76244885,-0.15420718,-0.04192792,0.32572323,-0.6471525,-0.5105301,-0.18548149,-0.6064673,0.3691234,0.32125938,0.20947473,-0.3494395,-0.21463212,-1.0152324,0.35222256,0.49581766,0.52718157,-0.3739313,-0.18341163,-0.4987254,1.1372547,-0.01565311,-0.45410958,-0.8340346,-0.00195411,-0.20779215,-0.3844096,-0.5327799,0.31612659,0.12760821,-0.37557688,0.44244835,-0.2663364,-0.22549431,-0.09283568,0.4546277,-0.43864346,0.02968271,-0.7985958,-0.6276926,-0.5816008,-0.29631257,0.55756795,0.23354226,-0.24318458,0.04323547,-0.7509241,0.17017166,-0.35040691,-0.4413889,0.11694613,0.10452295,-0.8175521,-1.0960413,0.29959357,0.18106651,-0.32202268,-0.43666413,-0.99224055,-0.5947665,-0.14055562,-0.5306,-0.63369316,-1.0933683,-0.01260735,0.83533305,-0.07009992,0.3199124,0.04107929,0.00421898,-1.1032156,0.1792508,0.22573908,-1.0796959,0.8907517,-0.23443529,-0.21525119,-0.3423701,0.14917074,0.03414589,-0.0716801,-0.52680343,0.16277286,1.3563039,-0.74360055,-0.3111594,0.14965238,-0.14765891,-0.5412551,-0.07852933,0.3301736,-0.19905378,0.44067118,0.19747251,0.61816454,0.00464878,-0.05918895,0.5596607,-0.19859007,-0.22980586,-0.01323724,-0.9234931,-0.06314345,0.17184177,-0.13989116,-0.0590673,0.30726075,-0.12642808,-0.11370575,0.30901244,-0.29675117,0.13962935,-0.08990242,-0.25727233,-0.2701752,0.18892547,-0.06050317,-0.11676801,0.23820396,-0.6389877,-0.48591623,0.14653896,0.22857113,0.09430139,-0.19708592,-0.10178763,-0.42802116,0.00609779,-0.24384694,-0.1962579,0.2821203,-0.07953141,-1.2344607,0.4616613,1.0616989,0.24844643,0.25545347,0.25882503,-0.23285769,0.12016268,0.6938299,-0.9110795,-0.12623681,0.06248473,0.43697852,0.20155622,-0.2429519,0.53116035,-0.31870365],[-7.51739919e-01,6.64367616e-01,4.59361315e-01,1.47443879e-02,5.36959589e-01,3.91232193e-01,-4.60711449e-01,1.45903006e-01,-1.20857574e-01,-7.96402395e-01,3.76522988e-01,3.23723316e-01,-3.93662989e-01,-2.34245464e-01,7.20820785e-01,-2.95715958e-01,-5.38056552e-01,5.24259090e-01,8.53514895e-02,1.13712952e-01,-4.21643287e-01,5.70245147e-01,3.77553515e-02,-1.81597963e-01,-4.72668111e-01,9.00509953e-02,4.90886122e-01,6.55739009e-01,-2.93581307e-01,-1.74301118e-01,-2.05917433e-01,-3.38932760e-02,-2.06028625e-01,2.05425601e-02,-1.05153525e+00,-2.18989775e-01,-4.26765112e-03,-2.82341629e-01,-1.06275761e+00,-1.74636289e-01,1.68556497e-01,-6.22259438e-01,-1.78970918e-01,-5.79753704e-02,-6.67205229e-02,4.27576423e-01,4.02686030e-01,3.96898955e-01,6.93216085e-01,7.72900209e-02,-1.49562620e-02,-7.66074717e-01,6.53867126e-02,-1.70569673e-01,-1.46809578e-01,4.55368847e-01,6.67640418e-02,-1.62745729e-01,-1.26826083e-02,5.89848936e-01,-1.61096767e-01,8.47309083e-03,4.43711609e-01,5.52739441e-01,-5.88465452e-01,6.97632492e-01,2.65916646e-01,-7.63353586e-01,4.50659320e-02,3.89282376e-01,-7.28938758e-01,7.57732475e-03,7.85785317e-02,-6.53555691e-01,-3.27120945e-02,-3.58235061e-01,-6.11960053e-01,3.46518070e-01,5.41276395e-01,5.31268716e-01,4.99355733e-01,2.03118652e-01,3.47924739e-01,-4.65967715e-01,-2.08704412e-01,-9.04022008e-02,-1.98231101e-01,-6.55176163e-01,-7.11800873e-01,1.23495571e-01,-8.72892022e-01,8.69296119e-02,2.90867656e-01,-1.17139854e-01,-9.51249897e-02,2.62186259e-01,-5.47365248e-01,7.83610463e-01,1.24393605e-01,-5.20791352e-01,-1.84584379e-01,6.38460815e-01,-2.92438567e-01,1.60676986e-01,-7.28828013e-02,-1.70135964e-02,-4.53472808e-02,-3.88702482e-01,6.96373940e-01,-2.87751704e-01,8.18624571e-02,-1.11214548e-01,5.88046789e-01,2.68919587e-01,1.55517876e-01,-4.69064087e-01,-2.58841515e-01,-5.92973292e-01,-7.79218197e-01,-5.05262017e-01,-9.70861554e-01,-3.09004277e-01,4.89014745e-01,-7.01250657e-02,-2.06380224e+00,2.24672079e-01,-9.68953669e-02,-1.20145440e+00,5.50294280e-01,-2.65048325e-01,1.95357442e-01,2.22619653e-01,-2.69873410e-01,-1.70908391e-01,-4.18944329e-01,1.41215667e-01,-4.18911912e-02,-1.49379104e-01,3.39682400e-01,-2.15118989e-01,-1.50600433e-01,5.37012041e-01,-1.09244514e+00,8.61005664e-01,-2.46369004e-01,5.76373756e-01,-3.72273684e-01,5.35806306e-02,1.70292661e-01,4.61153597e-01,-7.47330859e-02,2.39449888e-02,-1.76030770e-01,-1.17578328e+00,3.69268090e-01,-5.92120230e-01,-6.51851296e-02,-5.06655216e-01,-3.67434978e-01,-1.28776073e+00,-1.04183733e+00,3.00930530e-01,-3.57116014e-01,1.60971656e-02,1.89073399e-01,5.11222899e-01,-1.28077507e-01,4.27633077e-01,2.70100147e-01,9.77492556e-02,1.04990996e-01,2.81475812e-01,-4.25907165e-01,7.50086829e-02,3.46631259e-02,6.42756283e-01,-2.62189120e-01,-5.95984161e-01,-2.71534920e-01,-6.59543693e-01,-5.98966144e-04,6.51569963e-01,3.01049259e-02,-7.72746623e-01,-6.69503212e-01,2.04838455e-01,9.96998549e-02,4.45122391e-01,-3.13307762e-01,2.38327697e-01,4.03277844e-01,-4.19748694e-01,-5.68860352e-01,5.57468474e-01,-1.44144893e-01,2.41764337e-01,6.97342694e-01,8.92874897e-02,-6.45546496e-01,2.71156490e-01,2.36737251e-01,1.36846155e-01,-2.47364379e-02,-1.94448009e-01,6.12564623e-01,3.34761798e-01,3.46161634e-01,4.63947386e-01,-4.43479449e-01,-5.82470931e-02,-8.10248077e-01,1.64246812e-01,1.58538267e-01,-8.25969577e-02,5.47248483e-01,-8.75679731e-01,-5.87994158e-01,3.42831500e-02,2.18562350e-01,8.66832972e-01,-1.67922601e-01,-5.46559580e-02,4.31752801e-01,-1.17916510e-01,-6.32283092e-03,-3.97054076e-01,2.74659455e-01,4.06498909e-01,4.67603832e-01,-3.29848379e-01,4.82796073e-01,-5.64936772e-02,2.62373567e-01,6.73065111e-02,9.09647346e-02,2.93582559e-01,-3.22547615e-01,-2.22489744e-01,-2.34922215e-01,-1.27875376e+00],[-2.60987669e-01,3.58252734e-01,-1.86630368e-01,1.24094382e-01,-9.58408415e-02,5.20801902e-01,3.89563560e-01,2.78582036e-01,-2.77533799e-01,-2.06722513e-01,-5.38405061e-01,1.46785408e-01,2.06126481e-01,-3.82500917e-01,-3.79817545e-01,5.92023656e-02,5.79452395e-01,-7.94295192e-01,-5.10522649e-02,2.23756850e-01,3.87233257e-01,-5.95295608e-01,-2.82354653e-01,5.35801724e-02,-1.25094578e-01,1.26899943e-01,1.65467826e-03,4.46359739e-02,-5.04820086e-02,5.26285529e-01,1.70696825e-01,5.63877761e-01,5.51499724e-01,-4.58649620e-02,-4.78642851e-01,4.63135779e-01,-5.44798434e-01,3.56928632e-02,2.88549904e-02,-5.14786184e-01,-2.95985401e-01,-2.06302062e-01,3.60761117e-03,-8.12981948e-02,-8.30751434e-02,2.97381878e-01,-1.22615531e-01,-4.84711617e-01,2.58744359e-01,-6.28264109e-03,-9.38108563e-02,-3.86806965e-01,1.10916510e-01,-1.65940261e+00,-5.48817039e-01,-1.06890500e+00,-4.94146310e-02,4.57092226e-01,-1.34439513e-01,-2.79249847e-02,4.96195376e-01,-3.59782845e-01,5.72609901e-01,-4.90930229e-02,-4.72501189e-01,7.20072865e-01,-6.77533686e-01,3.67568713e-03,-6.59853444e-02,3.05883974e-01,2.46886700e-01,-3.89121950e-01,-5.82472146e-01,3.77280474e-01,-2.23019764e-01,-9.24043879e-02,-2.57211059e-01,1.56356141e-01,-4.05118078e-01,3.38378578e-01,2.02953056e-01,2.01617733e-01,1.69744670e-01,4.94655699e-01,-8.22281837e-01,-3.17802787e-01,1.49214819e-01,7.89410293e-01,-2.29303427e-02,-2.14804903e-01,-8.48033965e-01,-1.79739878e-01,-3.77254963e-01,-3.43300253e-01,-3.81843522e-02,5.89539766e-01,-2.81111091e-01,3.43704849e-01,-8.52526873e-02,-3.08062971e-01,-3.89373809e-01,-4.82456088e-01,3.03554654e-01,2.18980715e-01,3.24917704e-01,7.39540696e-01,1.78322986e-01,-4.08376634e-01,6.96203291e-01,-6.74024582e-01,-1.15789497e+00,-9.15973410e-02,2.67664194e-01,-1.16106100e-01,8.37896466e-01,-9.16399211e-02,2.00858325e-01,1.84474096e-01,1.44211531e-01,-1.38157547e+00,2.04491690e-01,-4.04987156e-01,-8.20448902e-03,6.25963330e-01,-5.56961857e-02,8.74277353e-01,-3.83916013e-02,-7.89530218e-01,5.90002120e-01,-3.09643567e-01,8.35217535e-01,-6.18282221e-02,-4.08613265e-01,6.86920434e-02,3.28486830e-01,-4.71742094e-01,-5.33957601e-01,-1.48863077e-01,-8.61386418e-01,-1.15605697e-01,4.83939014e-02,8.99110913e-01,-3.44915181e-01,-2.00820163e-01,-7.21908584e-02,1.50537521e-01,-5.32170117e-01,4.32544082e-01,3.58164489e-01,-9.59262013e-01,3.36829036e-01,-1.02595210e+00,8.09603751e-01,6.98134422e-01,5.00745714e-01,-3.47628355e-01,1.01279974e+00,-7.01849699e-01,-6.88273609e-01,-5.51650584e-01,2.22375482e-01,-4.80008125e-02,6.82964385e-01,-4.09296364e-01,-4.45023954e-01,-5.32360375e-01,1.37607362e-02,-2.52388120e-01,-2.08954006e-01,2.20807493e-01,8.12381208e-01,-3.15746032e-02,4.21039343e-01,1.52235880e-01,2.80933738e-01,2.30496585e-01,-8.13073754e-01,-3.98199052e-01,-3.37734461e-01,1.50116324e-01,4.73072976e-01,2.09573880e-01,-3.83967191e-01,-1.69699103e-01,-5.81070781e-01,-8.60152170e-02,-1.27383927e-02,-5.04028462e-02,4.93147284e-01,-6.98304474e-01,2.21392453e-01,5.56052327e-01,-2.94362873e-01,2.13791654e-01,-3.97652984e-02,6.59508705e-01,1.87800135e-02,-1.17034972e+00,-2.23822296e-02,-1.36202723e-01,8.94121468e-01,-4.98713911e-01,-2.60497630e-01,1.04558274e-01,8.73566940e-02,7.09745705e-01,3.49291898e-02,5.03904879e-01,-7.41081059e-01,-2.38134876e-01,6.12575889e-01,-5.93118012e-01,-3.61263722e-01,9.27272588e-02,-1.09400845e+00,2.54531860e-01,4.56625462e-01,2.52275895e-02,2.98221409e-01,-9.06624734e-01,-1.67465195e-01,-8.67859602e-01,-3.48748624e-01,6.01917982e-01,4.00024056e-02,4.62904453e-01,7.14076817e-01,4.31171864e-01,2.67535567e-01,-6.82959735e-01,8.38197693e-02,7.27931112e-02,-7.72177219e-01,-1.44662976e-01,3.95421565e-01,4.72614288e-01,1.88368559e-01,-1.79628700e-01,7.94464201e-02,-9.03905556e-02],[-0.28070927,0.5247147,-1.0657701,-0.2128082,0.3668556,0.44002965,-0.30678552,1.1451999,0.8049015,-0.20850708,-0.00519779,-0.07216715,0.7195762,-0.17933755,0.45196486,-0.1644369,0.17145865,-0.08027788,0.0325686,-0.10778768,0.20542061,-0.38582885,-0.2976987,-0.24772228,-0.7337642,0.63025576,-0.48267,-0.35153458,-0.50163,0.86278814,0.22410649,-0.37014222,-0.38681817,-0.18416627,-0.05635896,-0.37002558,-0.76585805,0.04060515,-0.21905996,-0.1828488,0.35850307,0.0023263,0.8809971,-0.6498254,-0.49601427,-0.3425871,0.09480125,-0.15689269,0.9483347,-0.11645546,-0.45613334,-0.04764784,-0.23004377,-0.15113467,0.35021827,0.3305033,0.3652549,-0.20693865,-0.12346357,-0.05144916,-0.62464625,0.26137286,-0.16746604,0.1766616,-0.1298364,0.4893744,-0.19265224,0.6723979,0.48730886,0.24860737,0.3318167,0.55760634,-0.7228618,0.8225809,-0.8871843,0.04602751,-0.5875426,0.24282576,-0.8275785,-0.19684969,-0.22581162,-0.38122252,-0.25542212,-0.4532805,0.09410777,0.105772,-0.36234722,-0.2291742,0.33773622,0.25900164,-0.07535519,0.37826043,0.38552758,-0.0873743,-0.23599428,-0.01884742,-0.06447542,-0.11527353,-0.6727864,0.03813864,-0.49159074,-0.42967033,0.21163261,-0.48171532,0.80107576,-0.43964842,0.59702396,0.03767547,-0.18139613,-0.40832683,-0.5525795,0.30440497,0.18046091,0.77398914,-0.3376801,0.67606926,0.27117702,0.265779,0.49496984,0.11976537,0.08319748,-0.69729227,-0.23842609,1.3686751,-0.5368206,-0.82092464,0.5069168,0.02354404,0.73433423,-0.10747197,0.06297084,-0.44397643,0.7365571,-0.30373764,0.01021118,-0.05107826,-0.7274834,-0.6440516,0.89894646,0.01751176,-0.36418793,-0.31859595,-0.17465092,-0.13929833,0.08347315,-0.7187926,-0.337349,-0.3825434,0.4341805,-0.21962072,0.22581661,0.1131058,0.12351233,0.10574216,0.04735129,-0.5124926,0.07513234,-0.10536301,0.4493598,0.08009004,-0.50587285,0.01272342,-0.6090528,0.6249259,-0.25537604,-0.58145785,-0.45017013,0.75879234,-0.5563348,-0.4216198,-0.7701422,0.7561478,-0.39668694,-0.40026498,0.23960745,0.04487954,0.6069487,-0.31664023,-0.6728885,0.74915814,0.05084609,-0.42758885,-0.37327507,0.02454331,0.28639597,-0.26707605,0.8476039,0.176042,0.08329123,-0.1066796,0.00779491,-0.1416755,-0.8838418,0.47301453,0.12001245,-0.20342556,0.7863876,0.01445725,-0.5595644,0.5103438,-0.38407278,-0.22719707,-0.2621253,0.00716373,0.37697858,0.2251526,-0.10405834,0.31480658,-0.1607883,-0.22710045,0.5769015,0.18233067,-0.041887,0.07758778,-0.617967,-0.44166282,-0.67980003,-0.34761032,0.0678536,-0.2710137,0.15587959,0.16485101,-0.6131402,0.39629972,-0.02421826,-0.01445137,-0.0786684,0.03140229,-0.43287706,0.4286525,-0.24556829,-0.4040931,-0.3266736,-0.680004,-0.37666717,-0.67364544,-0.10441118,0.04611645,-0.9343534,-0.9932803],[-7.45102614e-02,-3.36094528e-01,-2.81431496e-01,-4.76748049e-01,8.21041390e-02,2.47137219e-01,8.35919604e-02,7.99682960e-02,2.47358128e-01,7.39849955e-02,-7.95232296e-01,8.12220693e-01,1.92093328e-01,6.64095700e-01,2.08418086e-01,-5.69905579e-01,3.78549159e-01,3.46455157e-01,7.36514255e-02,-4.47683297e-02,-3.09700191e-01,7.78542757e-02,1.56384427e-02,-5.94870567e-01,-1.75188124e-01,-6.75999284e-01,4.13076133e-01,-9.27148461e-02,3.79733086e-01,-1.11369416e-01,-1.00444400e+00,-6.92528903e-01,-2.09516078e-01,5.03122151e-01,-1.26855955e-01,-4.57589030e-01,-1.81616709e-01,2.01840028e-01,-6.97701275e-01,-4.91740525e-01,2.88915336e-01,-4.46637750e-01,-4.39735442e-01,4.21356618e-01,2.26611435e-01,2.22317055e-01,1.88709050e-01,-2.49744534e-01,6.10850930e-01,-1.95071995e-02,4.49676514e-01,-7.32245862e-01,-1.43816724e-01,-4.34930354e-01,4.64271337e-01,-1.09438491e+00,4.95676160e-01,6.22408807e-01,-8.30399897e-03,2.92211175e-01,-1.67229563e-01,-2.57720739e-01,-6.36623263e-01,1.74599111e-01,-1.93070099e-01,1.65148973e-01,4.41047549e-01,-2.49125302e-01,-2.20312327e-01,4.42307711e-01,-4.88837004e-01,1.11966446e-01,1.10014312e-01,-4.77507830e-01,-9.08045322e-02,1.83789551e-01,2.41342410e-01,8.00100341e-02,-4.94235009e-01,4.32549864e-02,6.53322518e-01,-1.66639432e-01,-3.50462496e-01,-4.97299016e-01,-2.65305400e-01,1.70422584e-01,2.81631619e-01,-2.24150270e-01,-5.03099710e-02,3.54118586e-01,-1.53531075e-01,1.84112564e-01,-3.23081195e-01,-6.28195098e-03,-9.18449759e-01,-7.69783020e-01,8.41303289e-01,-5.57549655e-01,-5.20594001e-01,3.58756453e-01,-8.36509019e-02,8.98240209e-02,-1.88173056e-01,2.17152447e-01,-1.68149203e-01,3.96121114e-01,3.70015472e-01,-3.92636776e-01,2.77382404e-01,-4.87181216e-01,-1.49492800e+00,-2.80707061e-01,-3.35912853e-02,-2.82781124e-01,-4.22990322e-01,4.27017480e-01,7.03244507e-02,-4.77087498e-01,1.83071107e-01,4.03706372e-01,-9.29654658e-01,1.62594572e-01,-1.23411167e+00,-4.40772980e-01,-1.01742446e-01,-1.09158441e-01,1.17154740e-01,-1.10526219e-01,2.28442207e-01,3.58694583e-01,-1.03080976e+00,-8.67376775e-02,4.38017666e-01,-1.53564420e-02,4.86629695e-01,1.18116982e-01,-3.50622870e-02,-7.62962997e-02,-1.98772699e-02,-8.30153674e-02,5.83824277e-01,-8.32336903e-01,-1.58756340e+00,3.47582549e-01,-2.73394704e-01,-6.91870153e-01,3.32560718e-01,-7.66322196e-01,1.62425309e-01,-2.55993277e-01,-1.20694958e-01,2.62642950e-01,3.70426297e-01,2.80809015e-01,6.09760463e-01,-5.85473597e-01,-1.22203505e+00,-5.51449582e-02,-1.84392538e-02,7.78205246e-02,-3.64596605e-01,-4.31934506e-01,1.40657282e+00,7.12216675e-01,-4.65975851e-01,-1.54569700e-01,9.56362605e-01,-2.10140795e-02,-6.10271335e-01,2.38889918e-01,-8.63653362e-01,-4.86137658e-01,-7.60452688e-01,-2.29073450e-01,3.91796499e-01,-1.46886289e-01,1.04637770e-03,1.17443040e-01,6.05033994e-01,9.46447104e-02,3.30590457e-01,-7.61674717e-02,4.60189670e-01,3.00400965e-02,4.66155587e-03,7.27145970e-02,-7.99234509e-01,-7.26300001e-01,-6.28464103e-01,-1.07918359e-01,4.04599547e-01,2.64493108e-01,2.79602051e-01,9.87709388e-02,5.99995315e-01,-3.28028709e-01,-7.29970157e-01,-5.06049573e-01,-3.90165031e-01,9.50856134e-02,-4.14201558e-01,-6.80963844e-02,2.67094105e-01,5.25281541e-02,-2.30574131e-01,3.24067384e-01,-7.14442253e-01,-1.37710825e-01,-4.17275429e-02,1.50445238e-01,1.93265989e-01,5.55533588e-01,-9.16963518e-01,6.90896749e-01,-7.92418793e-02,6.31197870e-01,5.36884665e-01,6.46487474e-01,-4.71989959e-01,2.89417863e-01,-9.95517135e-01,-3.74492735e-01,4.99949306e-01,4.86446321e-02,5.67707360e-01,-3.03578883e-01,-4.36088219e-02,2.41159558e-01,7.31580928e-02,-2.43832693e-01,3.05553496e-01,-3.91088486e-01,-2.98553228e-01,2.76216894e-01,8.46271217e-02,4.04191554e-01,6.52123988e-02,-5.62533557e-01,1.18399046e-01,1.53049588e-01],[0.17415479,-0.13563842,0.16022149,0.15839994,0.03575281,-0.1765135,0.24416421,-0.5524397,-1.0115712,-0.6273253,-0.38059685,0.09217629,0.6893421,0.0721975,0.29438716,-0.25270075,-0.47623613,0.5321496,0.36053908,-0.3236385,0.07834473,0.02485663,-0.5515288,0.17095488,-0.721461,-0.03646413,0.12285937,0.6883878,-0.7317639,-0.35364407,-0.3752336,0.7857686,0.10395999,-1.0848626,-0.2519373,-0.14386794,0.35083348,0.06101131,-0.7336987,-0.02273495,0.6687527,0.66046566,-0.6360219,-0.5499326,-0.33993283,-0.358853,0.19098425,0.11204099,-0.52523446,0.22147131,0.8401996,0.48864135,0.36568284,-0.35795546,-0.19653022,0.45789477,-0.5478601,-0.20324966,0.07556128,-0.21241198,0.2696542,-0.01517394,-0.09718494,0.21269125,0.33681417,-0.02159493,0.65842825,0.02837409,0.21588868,0.70752966,0.7529965,-0.19221714,0.41025826,-0.18490475,0.24309345,-0.22331406,-0.59116393,-0.6259005,0.68137044,-0.7153143,0.13002463,0.1661994,0.46896926,-0.03215348,-0.43448007,-0.7217374,0.5685897,0.19871506,-0.27255735,-0.5564427,0.10290619,-0.6919531,-0.33476907,-0.2321845,-0.04067649,-0.7222816,-0.26660442,-0.02068848,-0.31837928,-0.05592935,0.05386594,-0.13565472,0.05762765,-0.92874247,0.00531791,0.39426598,0.18694192,0.84661454,-0.47071686,-0.40770465,-0.23666018,0.5636213,-0.17157188,0.47019345,-0.63714653,-0.16591802,0.48023376,-0.5964638,-0.18278144,-0.38990697,-0.48298487,-0.30651772,0.36027893,-0.18911053,0.05378722,-0.36535117,-0.22600186,-0.04455087,-0.05641857,0.02330352,-0.12106102,-0.33342993,0.6069092,-0.16964002,-0.08775288,0.6522466,-0.49397364,-1.3680153,-0.0273238,0.11975507,0.13860606,0.7071444,-0.15398765,0.04904716,-0.524031,0.14366406,-0.11523843,-0.86003876,-0.4711297,0.03689539,-0.08423979,0.45826483,0.78092444,0.05230681,0.45751864,0.36239824,-0.08672821,0.7640451,-0.20528087,0.41095138,-0.0244188,-0.20086949,-0.92196065,0.7502743,0.07910868,0.21684647,-0.19811471,0.33318028,-0.12188874,-0.55625105,-0.0052216,0.06528346,0.61016715,0.17464839,0.5495246,0.18607938,-0.18109523,-0.45844993,-0.06829572,0.27350125,0.35597923,-0.57819474,-0.06134887,0.5369494,0.41260186,-0.25891566,0.24153613,0.04917681,-0.41401973,-0.00325864,-0.24145657,-0.6942823,-0.81099015,-0.7906894,0.31487295,-0.11398198,-1.0157938,0.51358896,-0.12021542,-0.09387739,0.31811133,-0.54242927,0.5872104,0.15987146,-1.2435985,0.04457663,0.49517164,-0.4521158,-0.82329243,-0.61170197,-0.08218232,-0.4641945,0.0549053,0.6223514,-0.90474975,-0.08507861,0.1377991,0.5865031,-0.3554997,-0.13153252,0.49962893,-0.61143744,-0.0591755,0.10491689,-0.36649576,-0.03543111,-0.50809956,0.2523965,-0.7728329,0.6258238,-0.5242647,-0.8939751,0.17992552,-0.49653274,-0.3254751,0.78301466,0.30406135,0.47756183,-0.42796266,0.516371],[-8.00519824e-01,-1.38951108e-01,-1.57043487e-01,6.32170975e-01,-1.93233192e-01,1.76118433e-01,4.16057348e-01,2.65192002e-01,7.01843798e-01,3.21010321e-01,-7.13967383e-01,-6.73644915e-02,-8.88903558e-01,5.93575418e-01,-4.66493398e-01,3.83104354e-01,-3.33062798e-01,-4.28575464e-02,-3.49103391e-01,-6.88761994e-02,-1.34714735e+00,-2.30138332e-01,-3.63898724e-01,-4.59124535e-01,4.08175558e-01,-6.26535714e-01,1.18090108e-01,8.10313746e-02,-3.22474867e-01,9.42825302e-02,8.20456147e-02,-1.68654490e-02,7.55440772e-01,-4.77546930e-01,5.67925453e-01,-4.02892917e-01,-4.11786465e-03,-2.81190306e-01,-2.64482170e-01,2.79084176e-01,-1.82012171e-01,-5.41638970e-01,3.65738571e-01,3.43964070e-01,6.80323958e-01,1.12119548e-01,-2.30434507e-01,1.72536418e-01,-2.69883126e-01,-2.85419941e-01,-4.65170294e-01,4.48230088e-01,4.07886058e-01,-1.47112444e-01,-1.03800964e+00,-9.60190818e-02,5.78308105e-01,8.16884160e-01,5.18755913e-01,1.90642089e-01,-6.94269359e-01,9.69989121e-01,2.66991407e-01,-3.30918372e-01,7.55154014e-01,7.00687766e-01,-6.21864378e-01,-1.55546471e-01,-4.80526388e-01,1.71588317e-01,-5.08986890e-01,-6.54043257e-02,2.64198989e-01,-3.83004397e-01,5.36468700e-02,1.39087260e-01,-8.26282501e-01,2.04519153e-01,3.56567264e-01,-2.90557593e-01,1.78351328e-01,-1.28454733e+00,-8.06543112e-01,2.43854284e-01,8.04707944e-01,-1.92640141e-01,5.44794500e-02,-5.09437382e-01,1.04068033e-01,-1.58979014e-01,-8.11460793e-01,-4.11552191e-01,-4.30901825e-01,-2.69509196e-01,2.03002766e-01,-1.88667357e-01,-1.08883344e-01,7.73199558e-01,1.69554621e-01,-2.72216976e-01,8.63772407e-02,3.10001999e-01,-1.76090062e-01,-1.70217648e-01,-9.84520793e-01,3.00869765e-03,3.78079295e-01,5.73826730e-02,-4.07035023e-01,6.65240943e-01,9.43141401e-01,4.47172642e-01,-3.78221534e-02,1.00181073e-01,7.90064752e-01,1.29839003e-01,5.83231628e-01,-1.38797879e-01,-1.18701458e-02,5.34383237e-01,-5.14380515e-01,2.30572537e-01,-9.55033422e-01,2.87248135e-01,-3.50130022e-01,-4.75319773e-01,-2.66643077e-01,2.65937150e-01,-1.23723656e-01,4.37339455e-01,-2.56706290e-02,3.94731723e-02,-1.06298316e+00,5.47345757e-01,2.46349141e-01,2.54957080e-01,-5.82657516e-01,-5.12514412e-01,-1.53145596e-01,-8.44324708e-01,-5.64165831e-01,-4.94895548e-01,-2.07091749e-01,-7.12292045e-02,-1.92385972e-01,5.89016341e-02,-6.55227229e-02,-1.97271317e-01,-2.73460567e-01,-3.34118038e-01,-1.69053674e-01,3.39221060e-01,9.55943167e-02,4.59398955e-01,4.42589909e-01,-3.90392333e-01,5.70117868e-02,-1.19146240e+00,6.06753230e-01,9.91569310e-02,6.57421410e-01,-3.63479882e-01,1.51109129e-01,-4.80905712e-01,-5.98762780e-02,-6.87429011e-01,-4.57451567e-02,8.10819447e-01,-1.98289216e-01,-3.87419701e-01,-7.06038624e-02,4.07642908e-02,-9.94772017e-02,1.68383822e-01,2.73895532e-01,-2.27507092e-02,-5.18302262e-01,-6.68783560e-02,4.06142920e-01,-2.07646981e-01,-3.38237196e-01,7.82678604e-01,6.96375549e-01,-2.73276091e-01,2.21110091e-01,-8.78247619e-02,1.47901207e-01,-2.13762388e-01,-1.71982095e-01,-1.35600314e-01,-2.38721624e-01,-5.51827788e-01,1.82607248e-01,1.61891710e-02,1.58741653e-01,3.15147899e-02,-1.13131583e+00,-1.82142511e-01,-2.74130199e-02,4.11288142e-01,2.51074523e-01,-6.08377695e-01,4.06545438e-02,-3.77504289e-01,3.97000074e-01,-3.65086913e-01,1.56952351e-01,2.70111283e-04,1.64508983e-01,-1.81683198e-01,4.05336879e-02,4.27468531e-02,-4.57452357e-01,1.78035349e-01,2.22144768e-01,1.11006446e-01,-4.37811941e-01,2.52100170e-01,3.42429221e-01,8.35375339e-02,-2.73428649e-01,2.51251996e-01,2.21214324e-01,1.13972604e-01,-2.89645582e-01,4.19007868e-01,3.68314713e-01,-3.16483229e-01,9.20282781e-01,-5.44490218e-01,9.80741307e-02,5.36204167e-02,-5.11582792e-01,6.82125151e-01,3.18611064e-03,3.13979983e-01,-6.43797934e-01,-8.60667109e-01,-1.93169322e-02,-2.37633914e-01],[-0.27257332,0.03801443,0.2094708,0.50871074,0.26060477,0.31461218,0.5355756,0.6326638,0.37297803,0.81657976,0.56319356,0.2888267,0.31439272,-0.02424387,0.42849535,0.58349764,0.2613383,0.20401277,-0.5633345,-0.6841396,0.40178803,0.31097853,0.40902254,-0.4176568,0.56580055,0.85333896,0.36102432,-0.1072959,0.7463994,-0.25699845,0.7717762,0.41919383,-0.13668801,0.6147048,-0.342499,-0.24164504,0.24465534,0.49502093,0.21762916,0.5598443,-0.9202663,-0.01519721,-0.15157431,0.4323147,0.02703107,-0.20315328,0.4340007,-0.24358326,0.73555964,0.1511869,0.12472087,0.00717074,-0.08398205,0.2797172,-0.10641497,-0.16885802,-0.4106893,-0.18298683,0.27007154,0.3862296,-0.53558004,0.06308804,0.06903634,-0.15266839,0.6049207,-0.34962535,0.5923802,0.32757866,0.49692264,0.32356042,-0.01486215,-0.3654188,0.7148657,0.17195189,-0.10637698,0.03461571,-0.62748355,-0.4340407,0.15182935,-0.46091297,-1.0167139,0.0503223,-0.23490961,-0.01651968,0.07228998,0.16340385,-0.23818487,0.03331737,0.1612209,0.642604,0.15593928,0.29663616,0.2155208,0.46064404,-0.03052389,0.17610532,-0.16774218,0.2887052,0.65117663,0.42403036,0.23273705,-0.23866963,-0.07456648,-0.22960524,0.497747,-0.16187282,-0.30055296,0.2517174,-0.89744204,0.12590533,-0.48979658,0.57143056,0.86957866,0.73432064,0.1860998,-0.43253413,-0.00736284,0.49195504,0.6196459,-0.57331854,0.30548507,0.5020005,0.1829201,0.37420213,-0.5627451,0.16011573,-0.01813853,0.17565084,-0.23062819,-0.3261039,0.03665592,-0.36449412,0.10313471,0.07150671,0.11056121,0.13594295,-0.19628614,0.42235398,0.1639177,0.07859726,-0.11380714,0.58095413,0.49103096,-0.19253747,-0.46063188,-0.35850152,0.37627238,-0.31932843,0.01509254,0.00604196,0.19384754,0.50941,-0.21496955,-0.26646358,0.8012793,0.20639011,0.04174147,-0.8133619,0.1812938,-0.7798738,-0.16935508,0.26676384,0.26388928,-0.2648799,0.03810139,0.24577975,0.07055375,0.8036403,0.11490784,-0.3673943,-0.73573315,-0.35152885,-0.6723925,-0.33647758,0.23945805,0.29680046,0.22011724,0.22874604,0.31016797,0.08091811,-0.08250995,-0.27959287,-0.02574578,-0.40496108,-0.0061382,0.7769681,-0.71166354,-0.54766667,-0.2881804,0.44993654,-0.29012883,0.40529573,-0.02735956,-0.41317704,-0.07099356,-0.5871126,0.4059521,0.75076157,0.27316788,0.21986927,0.11803317,0.7444037,-0.36711273,-0.12754302,0.42352438,-0.6305667,0.10594384,-0.11294807,0.47454485,-0.04251952,0.4716051,0.3918831,0.5359786,-0.11891942,0.0567064,-0.3525302,-0.5040935,-0.51164615,-0.2024599,0.50681347,-0.08702783,0.7606387,-0.26413628,-0.4317109,1.0321928,-0.00618131,-0.14642285,-0.21119678,-0.27006602,-0.15136221,-0.47667056,-0.23203777,-0.12935701,-0.7350741,0.29067013,0.16152017,0.03517393,0.00591499,-0.21600547,0.44134396],[-0.1214912,-0.00454873,-0.34072608,-0.5991443,-0.08784205,-0.12647483,0.23418763,0.5777891,-0.24586488,-0.08327349,-0.05758583,-0.00197944,-0.6495804,0.6461489,-0.25726503,0.4033817,0.7550627,0.4141352,-1.4496307,-1.0982826,-0.09888686,-0.04107013,-0.05504671,0.01420907,-0.01767487,-0.3038145,-0.29674482,-0.37187582,-0.25356966,0.623055,0.49392235,0.22755009,-0.4193535,-0.06968904,0.5902841,0.64647895,-0.60143167,-0.4171214,0.37584552,-0.9668434,0.35813776,0.18846083,0.54920787,-0.03483879,0.8319929,1.0861771,-0.57557803,-0.48804653,-0.34689814,0.12859227,0.5950647,0.2757848,-0.33792055,0.02662161,0.48102587,-0.01080695,-0.1849425,0.10942965,0.45345634,0.8396062,0.04879393,0.4197149,-1.4149469,-1.2444177,-0.4180346,-0.20508282,0.18673101,-0.0262266,-0.6069197,-0.5486576,0.9291795,0.5034212,-0.1571843,-0.15299399,0.53334486,0.6167979,-0.5302136,0.31232616,0.31815496,-0.36711475,0.02145332,-0.19834019,-0.7541996,-1.3153456,-0.43942535,-0.06340541,1.0316044,0.9854642,0.25059435,0.14608786,-0.16888955,0.07038335,0.33806437,1.637976,0.03301132,0.04920617,0.01868091,0.02944086,1.3816731,0.07720013,0.28317067,-0.36015892,-0.07705549,-0.5014296,-0.2518489,-0.20018663,-0.24908361,0.4664326,-0.3622055,0.02270186,0.05834718,-0.27282235,-0.22785039,0.5113386,0.11464577,-0.19410096,-0.09978557,-0.21521562,0.12000104,0.0938803,-0.11931052,0.18562773,-0.64352274,0.08285908,-0.15517457,0.21394388,-0.46659818,-0.52542543,0.24709536,0.13247769,-0.1335236,-0.00274389,-0.4930255,0.02968817,1.099946,1.6194618,0.02273223,0.00266376,-0.13822748,0.4576821,-0.01238644,0.14506108,-0.07613899,-0.5029412,-0.5631568,-0.04767017,0.97150457,0.05986623,0.06301828,-0.34767777,-0.18781899,-0.26262823,-0.1447898,-0.4343602,-0.46273103,-0.45512393,-0.4673891,-0.21529724,0.00621523,-0.01796723,0.24267852,0.00931579,0.20892422,-0.43171078,-0.21146767,-0.09237501,-1.4800822,-0.27483976,0.11372945,0.01066997,0.37121463,-0.08509205,-0.45317292,0.10309671,0.40444374,-0.46813813,0.27628782,-0.38308126,-0.36354706,-0.72013426,0.7208799,0.04403523,0.00872985,0.07392914,-0.98984826,-1.2995764,0.4752754,0.32303402,-0.02669311,-0.11793897,0.09982788,-0.09476522,-0.6117849,-0.19732589,0.14402945,-0.45406464,0.06273536,0.08955676,0.02101642,0.43630484,-0.09661365,-0.02231013,0.65052414,0.3828646,0.03055836,-0.73687077,0.54394037,0.73786455,-0.15744473,0.5103148,-0.3520363,0.10162034,0.33478498,0.26115164,-0.4304258,-0.120522,0.06523201,-0.09245763,-0.53442657,-0.17046037,0.28829268,0.407612,0.2575104,-0.16025485,-0.03470989,0.23657882,1.0053064,1.2094741,0.20490901,0.26314247,0.01304979,-0.1522869,0.4964774,0.58698875,0.0073599,-0.00790455,0.09472606,-0.2514435,-0.0067796,0.20054296],[-9.02392030e-01,5.83148777e-01,3.74322206e-01,3.96771312e-01,5.46840578e-02,-3.18716794e-01,6.32952273e-01,-5.14076352e-01,-6.73201442e-01,-8.42904001e-02,-1.30679831e-01,-6.07554093e-02,-1.22381806e+00,1.50429204e-01,-5.85604370e-01,4.89655405e-01,-2.51808971e-01,3.25027645e-01,2.21738979e-01,5.86967766e-02,-1.47418365e-01,-1.87087759e-01,1.54367998e-01,-3.82022262e-01,6.00962155e-02,1.95981577e-01,-1.14153288e-02,8.66211593e-01,1.68710992e-01,1.71085633e-02,-3.43263857e-02,-3.30064684e-01,-2.42744178e-01,7.20590632e-03,4.22655344e-02,5.36174059e-01,-1.85149923e-01,-3.58545125e-01,-4.24779981e-01,2.51330405e-01,4.45808917e-01,1.11646816e-01,-6.86156452e-01,6.03407443e-01,7.84387589e-02,1.73742756e-01,2.79858876e-02,2.07809117e-02,-1.05859292e+00,1.82176068e-01,-1.44494385e-01,5.14327109e-01,2.32619062e-01,9.68973488e-02,3.53104562e-01,-1.94555372e-01,2.54988849e-01,2.13811353e-01,1.07700042e-01,1.05171606e-01,7.01006413e-01,-4.52921361e-01,4.71839249e-01,-1.87110960e-01,-3.21715713e-01,4.13795769e-01,6.50631309e-01,4.19030935e-01,4.26617339e-02,3.86627525e-01,-2.52864063e-02,-4.20073003e-01,-4.69305158e-01,-5.29505372e-01,-3.58206183e-01,-5.34261346e-01,1.12293579e-03,-2.68719524e-01,-4.52512115e-01,-6.66966960e-02,3.72049689e-01,4.30559248e-01,-3.64939302e-01,-2.10897535e-01,-2.97848880e-01,1.63256660e-01,-2.24763319e-01,7.57403672e-01,-2.02981547e-01,4.12773341e-02,-4.57346499e-01,-1.15159333e+00,-5.28843999e-02,1.96722195e-01,4.55925763e-01,9.30398703e-02,1.66843757e-01,5.19723408e-02,7.95590356e-02,6.58874810e-01,-9.76282880e-02,-6.71530217e-02,8.18853825e-02,1.78693995e-01,1.10278286e-01,1.42230347e-01,-5.27922332e-01,-7.62437046e-01,-3.84196043e-02,6.98530257e-01,9.50204968e-01,5.51099852e-02,-3.59050751e-01,-2.39739522e-01,-1.06821764e+00,-4.71777171e-01,2.63601124e-01,5.57628989e-01,4.15127367e-01,5.55001557e-01,-6.36091828e-01,4.31020819e-02,4.66791064e-01,5.95782936e-01,4.58845496e-01,2.52880991e-01,-6.86307192e-01,-1.10834777e-01,6.37549102e-01,1.23228751e-01,4.65910524e-01,-5.17232390e-03,2.07438350e-01,-1.38963491e-01,3.65211546e-01,-2.99112797e-01,-1.13400686e+00,2.82547772e-01,-1.80305195e+00,-8.33788574e-01,-1.81514740e-01,-3.24106365e-01,-8.90250325e-01,3.58230919e-01,-3.80122334e-01,-7.17996135e-02,5.85406601e-01,1.02215111e-01,-3.63514349e-02,5.92287257e-02,-5.97640514e-01,3.52654070e-01,-3.55683595e-01,8.11077714e-01,-8.40234220e-01,2.19566211e-01,-2.92761803e-01,-3.57830793e-01,4.42211062e-01,6.14219129e-01,4.90552187e-02,-1.01478808e-01,-3.01826745e-01,-4.30534124e-01,3.03276658e-01,-2.70132333e-01,1.08203828e+00,8.14722598e-01,-5.27338982e-02,-2.38720756e-02,9.03054550e-02,2.61395782e-01,-8.03310394e-01,-4.44242895e-01,-1.06575859e+00,3.82058948e-01,5.03485054e-02,-1.13497615e+00,1.49123490e-01,-6.77277923e-01,3.11686128e-01,-1.18451945e-01,-6.07117772e-01,1.45696893e-01,3.50816488e-01,3.56999576e-01,-2.77183443e-01,-1.19701815e+00,2.27862254e-01,5.00776410e-01,-3.76871884e-01,8.70395303e-02,1.70529202e-01,-4.24450845e-01,-2.98525214e-01,2.62391984e-01,8.66594389e-02,-1.60798267e-01,-2.18964607e-01,5.36964357e-01,-2.80917168e-01,1.30891800e-01,-5.64733744e-01,-4.29029346e-01,-6.44481719e-01,-2.38031894e-01,1.75127402e-01,-4.48690861e-01,-8.48700404e-02,7.98593313e-02,3.98789018e-01,9.55981389e-02,7.21688628e-01,-1.08156092e-01,-2.38015041e-01,-5.86129367e-01,-6.54994473e-02,3.18392903e-01,-2.22967386e-01,1.58007279e-01,-9.63293850e-01,5.63556731e-01,3.95896584e-01,-6.07358456e-01,2.77984083e-01,-2.42489547e-01,-4.34490800e-01,4.17894907e-02,-8.34749818e-01,-9.62722450e-02,5.53159833e-01,-4.32624929e-02,2.14686066e-01,7.89306164e-02,4.17629451e-01,-7.81567574e-01,4.31471199e-01,-3.27317089e-01,-1.14491701e+00,-5.13307631e-01],[3.04406911e-01,3.74412313e-02,1.06274389e-01,-6.29062355e-01,-3.78047526e-01,-4.39813644e-01,4.30218160e-01,4.74783808e-01,2.64237761e-01,2.13698342e-01,-6.05289459e-01,3.82277340e-01,-9.17667523e-02,-5.14258035e-02,1.36724293e-01,-9.26564038e-01,1.30210981e-01,-2.99531311e-01,-3.13171357e-01,1.38908595e-01,1.00727975e-01,1.98367178e-01,-6.89115107e-01,1.76588252e-01,-4.11005393e-02,1.45637795e-01,8.35701287e-01,1.03578366e-01,4.27203238e-01,-3.82595778e-01,-3.59244168e-01,-1.81466773e-01,4.27529216e-01,2.75539786e-01,-8.65754485e-02,-1.66203022e-01,-1.54100880e-01,-4.86310683e-02,1.09119266e-01,-7.66876221e-01,-2.96515554e-01,-1.36819363e-01,-4.09443498e-01,-2.16137081e-01,-2.10521162e-01,-6.55398905e-01,-6.60327554e-01,4.68973249e-01,-4.26136941e-01,2.06465095e-01,2.15990812e-01,-1.10626984e+00,-3.09732437e-01,3.69483948e-01,-4.73475605e-01,-2.54234392e-03,-4.02467191e-01,2.36737341e-01,-5.34729779e-01,4.19329762e-01,-1.59939956e-02,-7.26385713e-02,5.72235174e-02,-1.54590935e-01,1.88216612e-01,9.69744176e-02,-5.47702014e-01,1.81090042e-01,2.24745050e-01,-3.06192130e-01,8.09249133e-02,5.35276681e-02,4.02732968e-01,-2.72380531e-01,5.09210408e-01,-1.04444778e+00,-5.49022019e-01,-3.78210813e-01,1.02736443e-01,-1.38020113e-01,-3.25041831e-01,2.95453727e-01,-9.51735377e-02,1.37709767e-01,-8.33470047e-01,-1.85337633e-01,-1.51540399e-01,-1.05812095e-01,2.91781843e-01,5.34107327e-01,-3.31745416e-01,-3.95550951e-03,3.65939945e-01,3.84860277e-01,-1.20340407e+00,-5.03256440e-01,2.90971041e-01,5.26458621e-01,-2.89809465e-01,1.28070727e-01,-4.43345189e-01,9.66220140e-01,-4.42868531e-01,2.67361999e-01,-2.88605720e-01,3.66548836e-01,-1.74644426e-01,3.61212224e-01,1.82195902e-01,-8.77859771e-01,-2.09839255e-01,-2.59123504e-01,-4.70273405e-01,6.01709366e-01,5.04730880e-01,-4.36019003e-01,1.92812398e-01,-5.73569417e-01,5.15259027e-01,-1.01474905e+00,-2.43537590e-01,-3.74887109e-01,-5.11894077e-02,8.83568466e-01,-8.49575818e-01,-2.61955410e-01,2.23809928e-01,5.53922415e-01,-4.56873417e-01,-5.08797109e-01,2.44707003e-01,-1.50418270e+00,4.67842728e-01,-6.19154155e-01,-9.48800445e-02,-2.68923193e-01,3.42370600e-01,6.47801757e-01,-3.35620195e-01,2.57888716e-02,-2.82264054e-01,-9.26356986e-02,5.55080250e-02,-1.06262004e+00,-1.85852677e-01,-6.94187284e-02,-2.02706754e-02,6.37463987e-01,2.05158159e-01,-4.39921379e-01,-7.51485601e-02,-8.70905817e-01,1.28388330e-01,1.78538531e-01,-1.17500946e-01,1.54491976e-01,1.23131044e-01,6.54202461e-01,4.95492220e-01,-4.37649041e-02,-5.32839775e-01,4.00007546e-01,4.28158402e-01,7.98616290e-01,-1.61989015e-02,7.07954988e-02,-8.12388361e-02,9.04949307e-02,6.63761199e-01,-3.91904294e-01,-6.23685792e-02,-1.22765839e-01,-2.51402818e-02,-1.20890224e+00,1.55311320e-02,-2.09741905e-01,-3.02411646e-01,-3.69044363e-01,-1.69040442e-01,5.75739563e-01,-5.61112314e-02,5.57201207e-01,4.48791116e-01,1.27007097e-01,5.90992421e-02,5.05109489e-01,-3.33807677e-01,1.96632445e-02,3.65541607e-01,-1.99440159e-02,-1.17549884e+00,1.39053096e-03,5.96447945e-01,4.25438017e-01,-2.66619235e-01,4.15366292e-01,6.93275630e-01,4.11233336e-01,1.00939460e-01,1.94758967e-01,-1.04912555e+00,5.15081346e-01,3.51426043e-02,-9.20497715e-01,-1.37727663e-01,-4.58400510e-03,2.11209074e-01,4.33098435e-01,-6.85422003e-01,1.40787289e-01,-1.49143398e-01,-2.52571434e-01,-4.56987828e-01,-4.98271286e-02,-3.67367603e-02,6.97568417e-01,-3.56074333e-01,4.55011934e-01,-2.07884103e-01,-5.09018123e-01,-1.55299932e-01,1.50940430e+00,-5.72537661e-01,1.89972952e-01,-6.22354686e-01,2.04863459e-01,3.10589105e-01,2.86871016e-01,3.47300500e-01,4.60683644e-01,5.51700871e-03,-3.04232508e-01,1.76853165e-01,-4.43335772e-01,9.40452933e-01,1.18135484e-02,-2.04213902e-01,-4.12039250e-01,-4.36734676e-01,1.65172979e-01],[5.1578069e-01,-5.0337490e-02,1.4340799e-01,-1.6785918e-01,4.5105737e-01,2.7113780e-01,-7.3360890e-01,9.2965424e-01,-4.4766700e-01,-3.2656369e-01,3.1571227e-01,4.2082611e-02,3.4734097e-01,-8.9585625e-02,-4.4011477e-01,-1.9128342e-01,2.3785257e-01,-1.1579221e-01,9.0573370e-02,-1.3440241e-01,-5.0240737e-01,1.5215117e-01,2.8384054e-01,-5.6196696e-01,-3.3843654e-01,6.0988098e-01,-8.1701977e-03,-2.2919764e-01,-2.3252720e-01,-7.7908784e-02,2.2518056e-02,-1.9406232e-01,-3.3452708e-01,7.9233152e-01,-1.0136992e-01,1.6910072e-01,-1.6979180e-01,-5.1150344e-02,5.8651775e-01,-3.3825412e-01,-4.1022471e-01,1.7005621e-01,5.6755459e-01,1.8435264e-01,7.5661547e-02,-1.4606398e-01,4.4526437e-01,1.9036579e-01,7.5654817e-01,3.3498716e-01,3.1185210e-01,-5.8911556e-01,5.2564973e-01,-3.2779202e-02,-1.1026219e+00,-7.7320033e-01,-7.3535621e-01,-7.7106434e-01,1.7250329e-01,1.4775284e-01,5.9549391e-01,-1.8633629e-01,-5.6685776e-01,-2.6177442e-01,1.0051056e-02,-1.1719521e-01,3.0823371e-01,9.2198357e-02,-6.3906908e-01,6.7077315e-01,1.8954997e-01,-7.3617928e-02,-5.6373551e-02,-7.2543822e-02,-1.8557353e-01,-4.2454672e-01,-3.5299918e-01,-2.7021263e-02,9.7813241e-02,-8.5267907e-01,5.2334857e-01,5.9800029e-01,4.3463877e-01,-5.5195087e-01,5.5690491e-01,-3.9183792e-01,-7.9749964e-02,2.1387050e-02,-1.3422281e+00,2.9824060e-01,-1.8966600e-01,3.5581419e-01,-3.7459213e-01,-7.4824490e-02,-5.5438112e-02,2.4200067e-01,-1.1451898e-01,-2.4945056e-01,4.9883217e-02,-1.2617669e-01,1.1825606e+00,4.7088686e-01,7.3921056e-03,1.9540739e-01,-1.5462555e-01,6.6338724e-01,-4.2410070e-01,1.9086294e-01,5.3295702e-01,3.3923611e-01,-1.4339061e-01,-3.5210955e-01,2.3353738e-01,2.8996542e-01,-3.7341347e-01,-3.8313794e-01,-5.1418400e-01,-1.7428843e-02,-1.8033731e-01,-1.6800617e-01,2.3824663e-01,-7.2423643e-01,3.8064519e-01,5.8270198e-01,-1.5710753e-01,-1.7847191e-01,-3.9703022e-03,2.5060177e-01,7.3992476e-02,1.6431215e-01,-1.8529838e-01,9.6271693e-04,-3.3125901e-01,-7.4276924e-01,-1.0810427e-01,4.7054216e-02,-4.4914719e-01,-2.1751529e-02,-9.9834114e-01,2.1415221e-02,2.1811511e-01,-9.5443767e-01,-1.7922558e-01,7.3714316e-02,-5.2018142e-01,3.2104567e-01,-8.2653999e-02,1.9762976e-01,6.8416762e-01,-9.8734605e-01,1.8115258e-02,3.2300618e-01,-6.1008823e-01,-1.3742414e-01,7.6630563e-02,2.8771743e-01,6.8757486e-01,-7.6815438e-01,6.8867482e-02,3.9859071e-01,-1.6186250e+00,-3.1390432e-01,9.1842286e-02,1.0861307e-01,-2.2241299e-01,4.9934974e-01,2.9218671e-01,-1.0463227e+00,-5.3243256e-01,-5.7304587e-02,6.5133554e-01,-1.4924291e+00,-6.7652631e-01,-3.2935083e-01,4.4809815e-02,-6.8720031e-01,-9.0666324e-02,-3.2720780e-01,-5.5199206e-01,6.0196048e-01,4.9234357e-01,7.2559655e-01,2.4143414e-01,-7.8666158e-02,-3.6369976e-01,-2.1947289e-01,3.4539893e-01,2.1170154e-01,-3.0023533e-01,-1.1985872e-02,-8.9941090e-01,-5.3987491e-01,1.5188353e-02,-4.0206254e-01,-3.9351955e-01,5.0266588e-01,-1.2488770e+00,8.0933857e-01,-8.2736224e-01,5.6743413e-01,1.6142176e-01,-1.6010424e-01,4.1809124e-01,4.0076217e-01,-4.7295278e-01,-5.6217933e-01,-3.9507020e-01,-1.3627112e-01,2.7931008e-01,5.3820711e-01,6.2193286e-01,3.3539715e-01,-3.3619249e-01,-1.1785831e-01,6.1116755e-01,-1.1914511e-01,-1.6687587e-01,-1.6180584e-01,-1.8866648e-01,-6.9450039e-01,-3.2838073e-01,3.6180431e-01,-3.5606194e-01,-8.3510196e-01,-3.3821398e-01,3.3109143e-01,-6.2372819e-02,-1.0134434e-01,6.3992482e-01,-6.4600214e-02,-4.1590607e-01,-2.4823818e-01,-2.9010546e-01,5.6766337e-01,5.3951424e-01,4.1691963e-02,3.7005895e-01,5.4635242e-02,-1.2710708e+00,-4.7591440e-02],[-0.2281877,-0.08988646,-0.06595361,0.33991796,0.38956624,-1.1290361,0.28760624,0.5740528,0.3669914,0.20299919,0.48504972,-0.56026274,-1.0743384,0.18779793,0.54547966,0.5336578,-0.00788239,0.24678442,0.09637692,-0.14194226,0.1374751,-0.07261346,0.1402084,-0.41556296,-1.4654021,-0.8365357,0.46466246,0.26923606,0.06285482,0.24751648,-1.0263175,0.40781015,0.30907857,-0.2585985,0.8319375,-0.18193646,-1.4501972,0.07103621,-0.01191552,-0.73078,0.35176572,0.30546725,0.2893391,0.05877857,0.11064018,-0.34952345,-0.6995231,0.02551508,-0.8701454,0.15523735,-0.00489151,0.08943215,-0.09706922,-1.1970654,-0.6556272,-0.9009282,0.529641,-0.6267283,0.36203316,-0.1394133,-0.5159688,0.28141743,0.111133,0.09756063,-0.13096757,0.15172432,0.3191518,-0.6587441,0.589137,0.35043246,0.24859348,0.74696565,0.3186488,0.5706924,-0.35322544,0.05173399,0.4065543,-0.7505618,0.12428106,0.03735414,0.18663114,0.11254664,-0.5541063,0.12172701,0.28134468,0.88610864,0.01388335,0.07462306,-0.769954,0.302559,0.14924471,0.6351536,-0.20118587,-0.04514572,-0.02115308,0.84923416,0.6279386,-0.33769548,0.09894317,-0.35563177,0.6840389,0.5037152,-0.27670375,-0.13510755,0.10946706,0.02052538,-0.38511756,-0.6101007,0.23934753,-0.08364359,0.0449043,0.19348139,-0.5150609,-0.36114544,0.41857702,0.07814571,-0.05354586,-0.15531594,0.37957042,0.8557578,-0.02635635,-0.19010237,0.24813142,-0.516926,-0.40157545,-0.52484226,0.14407282,0.07874511,-0.08348618,-0.1906581,-0.14512704,-0.15457784,-0.27057263,-0.14275433,0.10016829,-0.34388626,0.10221761,0.5170786,-0.67657673,-0.2302263,0.19345759,-0.24391167,-0.14961396,-0.72803324,-0.60736656,-0.25786206,-0.59419847,-0.05019724,0.10981076,-0.30173242,-0.175446,-0.1500439,0.10602097,0.15228538,0.15311274,-0.41504842,0.2884159,-0.25507835,-0.9574182,-0.19014896,0.15303974,-1.6176947,0.38756314,-1.3075628,-0.20947452,0.5425439,-0.4765787,-0.04309433,0.24899925,0.5097251,0.5996564,0.3555529,0.36082312,1.089728,0.11463013,0.29449975,-0.37769043,-0.41131315,-0.11229874,0.00412139,-0.39696464,0.42475438,-0.68042815,-0.15152358,-0.52960646,-0.19368406,-0.9449168,-0.0659624,0.07039602,0.16782574,0.39656293,-0.24030526,-0.23434243,0.58603454,0.29904243,0.686533,0.6374544,-0.5789484,-0.01283436,-0.18299378,-0.292992,-0.11874688,-0.86865765,-0.0270747,0.24745078,-0.46372616,0.6234778,0.44883013,0.2360121,0.48122874,0.38193586,-0.39418104,0.32786494,-0.41793782,-0.39112765,-0.41656795,-0.21241152,0.05250662,0.3097362,-0.59107,0.9685697,0.2892938,-0.80742645,-0.3059806,-0.53106123,-0.1602776,-0.18238913,-0.19117655,-1.391989,-0.00518339,0.1932495,0.18127221,0.21005467,-0.61271566,0.15750206,0.238743,0.18399186,0.09007594,0.1121909,-0.93499327],[-1.5251797e-01,4.5591190e-01,-4.9425215e-01,-5.8728892e-01,2.4919418e-01,-5.7139294e-03,1.6632482e-01,5.5532938e-01,-2.2601876e-01,-2.3168603e-01,1.4664540e-01,2.4971931e-01,-3.0429617e-01,3.5669625e-01,-2.5802389e-01,2.7898183e-01,9.9537915e-01,5.4939634e-01,-1.1315281e+00,-9.3381673e-01,-2.0478433e-01,6.3127931e-03,-3.4014913e-01,2.6753864e-01,-1.3829859e-01,-1.9996846e-01,-1.9262275e-01,-2.8609565e-01,-3.7689641e-01,5.0525910e-01,5.8522874e-01,3.4695658e-01,-3.6610362e-01,7.5703591e-02,5.0414944e-01,2.1150747e-01,-2.2822975e-01,-5.3487217e-01,2.2886147e-01,-7.4696481e-01,2.2908147e-01,1.2931113e-01,7.1192944e-01,-1.3267076e-01,4.3748158e-01,4.1763404e-01,-6.8579668e-01,-6.2961483e-01,-2.5788212e-01,1.1724121e-01,4.3855134e-01,4.9276742e-01,-1.6426116e-01,-4.7641400e-01,2.6649383e-01,-5.7887688e-02,-1.7225283e-01,3.5768235e-01,-1.1398672e-03,2.4083468e-01,5.8755916e-02,4.4087917e-01,-1.2247257e+00,-1.0662367e+00,-4.1328475e-01,-1.7507744e-01,1.0321933e-01,-2.8449124e-01,-2.8789023e-01,-3.2615894e-01,9.3817234e-01,7.8634493e-02,-1.3366260e-01,2.2686546e-01,1.0546668e-01,-7.0500962e-02,-1.8112910e-01,2.9685527e-01,2.0459116e-01,-2.8374335e-01,-7.7766947e-02,9.2349267e-03,-5.2236664e-01,-1.2259215e+00,-2.8882939e-01,-1.1211776e-01,1.0505795e+00,1.0710655e+00,1.2195679e-01,-1.4688061e-02,6.9523160e-03,2.1439132e-01,4.8483774e-01,1.5682907e+00,1.5044343e-01,-2.0286654e-01,7.7406555e-02,-2.0416592e-01,1.0071654e+00,-1.3486710e-01,4.0593764e-01,-3.4457952e-01,-2.1520935e-01,-4.4655958e-01,1.8211974e-01,1.1105109e-02,-5.7569259e-01,6.1464769e-01,-3.9490998e-01,2.7516605e-02,-4.6942551e-02,-2.4447358e-01,-2.9767793e-01,5.5906647e-01,3.5967043e-01,-3.8673782e-01,-4.2356539e-01,-5.3441335e-02,6.8855211e-02,1.2819588e-01,-6.4782247e-02,3.6744830e-01,-3.4946427e-01,-2.6774791e-01,-8.7024681e-02,9.4091892e-04,-3.8395694e-01,-3.9974037e-01,2.2268938e-01,1.3222535e-01,-1.8323851e-01,-1.9141039e-02,-3.0079007e-01,1.5166396e-01,1.2094526e+00,1.2780395e+00,-1.6567424e-01,1.7986359e-02,1.8536238e-01,-1.4195656e-02,-2.1029701e-02,-1.6924740e-01,-2.0610711e-01,-3.1285852e-01,-4.8242834e-01,-2.6427197e-01,1.0717756e+00,1.4132683e-02,6.1615780e-02,-6.5024644e-01,-1.3604683e-01,-4.0097794e-01,4.1359574e-02,-9.6736901e-02,-2.7314046e-01,-4.0023151e-01,-1.7489401e-01,-2.4428520e-01,1.7334290e-01,1.4323536e-01,3.0774271e-01,-8.4153414e-03,-2.3196751e-02,-4.8374829e-01,2.4839838e-01,-9.5715299e-03,-9.2314374e-01,-9.8461092e-02,-2.4898443e-02,6.7499122e-03,2.0774795e-01,-4.8904937e-02,-2.8266934e-01,1.9011363e-01,2.9910567e-01,-5.1907432e-01,1.9910304e-01,-3.9391908e-01,-1.7952435e-01,-1.4643206e-01,6.7621410e-01,3.4608349e-01,-9.7417153e-02,-2.3457792e-01,-9.3522066e-01,-1.2105495e+00,2.0307133e-01,7.8292650e-01,-2.6465876e-03,3.8183644e-02,1.6943534e-03,1.0317091e-01,-6.7902005e-01,-1.4329737e-01,2.1201648e-01,-1.0871697e-01,-1.4552510e-01,-2.2403605e-01,-9.4087578e-02,2.9574707e-01,2.0943633e-01,-2.0449723e-01,1.8833154e-01,9.7460084e-02,5.2397209e-04,-7.9372180e-01,4.8555470e-01,4.1168678e-01,-3.3528894e-01,5.1511949e-01,-8.6570650e-02,3.0458260e-01,2.6736611e-01,1.9654696e-01,-2.6693350e-01,-1.4302798e-01,1.3688599e-01,-1.0470999e-01,-5.4098523e-01,-1.0756026e-01,2.0361698e-01,3.0275846e-01,3.1805399e-01,-3.3744931e-01,-2.9006645e-01,-5.6325532e-02,6.1113083e-01,1.1682658e+00,2.9465327e-01,1.8986630e-01,-3.0301985e-01,-2.4974287e-01,3.7523144e-01,1.6193041e-01,-2.5897312e-01,-2.1415225e-01,-8.8174447e-02,-1.7015179e-01,1.2825480e-01,-4.2941649e-02],[-1.39306292e-01,-8.39804232e-01,3.17531019e-01,-3.37640047e-01,-1.99827760e-01,-3.25734496e-01,3.90498221e-01,-8.81179810e-01,1.50878310e-01,-3.71930227e-02,3.96000624e-01,3.69593240e-02,-1.13295056e-01,-2.02465151e-02,1.64041564e-01,5.31361103e-01,-3.71711366e-02,-6.80661917e-01,-1.62841991e-01,-2.16506764e-01,-8.15485179e-01,-7.11084008e-01,1.01111598e-01,-4.58446592e-01,2.95148134e-01,-3.94246042e-01,-1.04610670e+00,-1.55692697e-01,9.67063963e-01,-2.07428366e-01,2.19336841e-02,-1.45834416e-01,1.88607499e-01,-4.27036405e-01,-5.93859315e-01,2.66857535e-01,-3.48168731e-01,-1.95675418e-01,2.74433512e-02,-1.41792357e-01,-3.91073763e-01,-2.39079729e-01,-4.77455050e-01,-5.44521570e-01,-1.10645130e-01,-7.72045851e-02,1.29536688e-01,3.02548677e-01,-2.27449745e-01,7.52531216e-02,-2.51253754e-01,-7.03220546e-01,2.21729338e-01,2.07277060e-01,9.20857936e-02,4.42754894e-01,1.58165231e-01,-7.95656562e-01,3.85010034e-01,4.27570164e-01,-1.28887862e-01,-5.55861533e-01,-5.16679734e-02,1.31272361e-01,-5.61803639e-01,3.21201921e-01,1.85601577e-01,8.61631706e-03,1.34755880e-01,-4.93604004e-01,-3.54079083e-02,5.46158731e-01,-7.21642196e-01,-1.32834747e-01,7.67418221e-02,2.82403439e-01,-5.81999481e-01,1.40063286e-01,2.18883559e-01,4.83579755e-01,2.23699048e-01,-3.83671612e-01,7.14716986e-02,-4.04939830e-01,-4.84639138e-01,-9.36944187e-01,-7.75134563e-02,1.28934264e-01,-3.19054537e-02,3.15070391e-01,-1.26834154e-01,5.97305834e-01,1.26416206e-01,-1.65437132e-01,-2.54318893e-01,5.95827341e-01,2.70989120e-01,-3.23783129e-01,-4.83771935e-02,1.37403712e-01,-5.20613492e-01,1.68359429e-01,-8.05922925e-01,8.72656107e-01,-5.39217442e-02,-2.57897913e-01,2.26534173e-01,1.36393690e+00,2.54777521e-01,4.01655555e-01,5.96027195e-01,-5.52057385e-01,-6.14096701e-01,-3.53474498e-01,1.67200491e-01,-2.34471768e-01,-5.86091101e-01,-5.53971946e-01,8.40628505e-01,9.65631083e-02,2.81661868e-01,-4.74049121e-01,-2.51321256e-01,-1.33877888e-01,-2.31464699e-01,-9.60499585e-01,-1.70602165e-02,-3.19831610e-01,8.60101879e-02,-3.71626318e-01,-9.29206252e-01,2.19414802e-03,6.60139203e-01,2.91049957e-01,1.36746496e-01,1.59412786e-01,-8.02105367e-02,-8.97841603e-02,-2.51952171e-01,-3.31622697e-02,3.39610487e-01,4.41897422e-01,-5.97727776e-01,-3.59038562e-02,4.63970035e-01,7.41317391e-01,-7.15943873e-01,-3.11439842e-01,-3.77862036e-01,-6.77472502e-02,-9.37147811e-02,3.01527996e-02,-1.56033069e-01,-6.57817870e-02,5.65925717e-01,4.24228013e-01,-5.40930986e-01,-7.57016003e-01,3.19649465e-02,-2.17325017e-01,6.83914900e-01,-4.94666807e-02,-1.27836692e+00,-3.22224259e-01,1.58211902e-01,-4.91076469e-01,1.11380652e-01,-5.21560788e-01,3.43394391e-02,4.04803753e-01,2.95679659e-01,5.63535213e-01,4.13604289e-01,2.67340660e-01,6.55571342e-01,-4.53061014e-01,2.53225565e-01,-2.71229029e-01,3.88934859e-04,-1.77189410e-01,-6.68394208e-01,-2.76517540e-01,1.28696617e-02,2.55080551e-01,-2.55232751e-01,9.04627144e-02,-6.02910876e-01,6.94530964e-01,-1.43905327e-01,-1.85693011e-01,6.12770140e-01,-2.16437817e-01,-1.25012660e+00,-1.19814947e-01,7.28255689e-01,-3.96227747e-01,-6.44306540e-01,-6.30418003e-01,2.88492143e-01,2.15489745e-01,-1.22383051e-03,6.57125354e-01,2.47622073e-01,-3.60677600e-01,2.54933327e-01,1.71061441e-01,-3.54270756e-01,-6.27172887e-01,7.24486768e-01,-1.37654102e+00,-3.51840667e-02,4.39301729e-01,2.98921317e-01,-6.13462627e-01,3.81753862e-01,5.52602589e-01,4.11105186e-01,-3.28521758e-01,7.37784326e-01,2.26821929e-01,6.35362044e-02,-1.29466012e-01,4.50321853e-01,3.89068685e-02,-5.58032513e-01,-9.82277319e-02,-7.58366287e-02,-2.20478669e-01,-9.58046496e-01,-4.40152496e-01,4.58665401e-01,6.65716380e-02,-1.32058099e-01,-6.34914100e-01,1.71111181e-01,-2.64995724e-01,-7.55798042e-01,6.93516254e-01,6.21879697e-01,4.59486991e-01],[-1.1340183,0.6337558,-0.14189552,-0.08834841,0.01390744,0.26007456,-0.320576,0.41082984,-0.91099906,0.03799701,-0.15391773,-0.26407722,0.23493879,-0.4048207,0.32275197,-0.34224042,-0.69492304,0.06421544,0.09983169,-0.19148117,0.36671957,-0.34951016,-0.4622857,-0.02961415,-0.05837281,0.07526045,-0.26519594,0.54581326,-0.56183755,-0.14379038,-0.7519155,-0.508857,0.44055876,-0.06003093,-0.05596523,0.10243172,-0.5384042,-0.23277506,-0.06589872,-0.58070254,0.23871732,0.08032843,-0.09878411,0.12247281,-0.21990459,-0.14320794,-0.10285905,0.00273292,-0.5952086,0.754318,-0.22329967,-0.3120038,0.36805144,-0.12848026,-0.15484649,0.82592505,0.06598881,0.3362285,-0.15545036,-0.31913263,-0.94640124,0.04658897,0.36773288,-0.18317132,0.36903083,0.45134568,-0.36606556,-0.38180634,-1.0376648,0.28141436,0.574981,0.50096977,-0.15416807,0.37448075,0.23574863,0.2466939,-1.4300181,0.13701017,-0.261021,0.00980909,0.1587048,0.36123,-0.6311063,0.42188698,0.4980933,0.04422487,0.3644541,0.2152602,-0.00606261,-0.78964674,-0.23395967,-0.49363938,-0.36866394,-0.32472113,0.9232047,-0.10874404,-0.08749907,0.509943,0.26310664,-0.15383899,-0.83928376,-0.66196674,-0.18663669,0.15224835,0.33017737,1.1115835,-0.01817985,-0.6506439,0.05114357,0.33595794,0.4126639,-0.691252,0.5065046,-0.15821266,0.10220842,0.3641345,0.7088726,-0.55372196,-0.17234975,-0.27682838,-0.60489583,-0.34033984,0.9134035,0.10307974,0.1183725,0.09315167,0.09262773,0.12513414,0.36838046,0.34395537,-0.50956184,0.13600634,-0.39084342,0.00358645,0.15185644,-0.38101855,0.27873832,-0.3024544,0.4767158,0.03621387,-0.49738833,-0.19157071,-0.38089576,-0.30335304,-0.63175195,0.32397854,-0.05948733,-0.01561071,0.1874487,0.41424406,0.64412177,-0.23877305,-0.7778653,-0.5090612,0.15218459,-0.02481386,-0.9805139,-0.7692799,-1.2199751,-0.3223925,0.44188344,-0.16959107,0.569569,-0.10125749,-0.758131,0.5271971,-0.9814201,-0.41517982,-0.06036297,0.56326544,-0.6105514,-0.49839213,0.35092777,0.86098963,0.42803606,-0.73178464,-0.696919,-0.6767456,0.36978272,0.49748513,-0.24684449,1.0365694,-0.93063366,-0.62790304,-0.7861015,-0.2682574,0.09331388,0.39312285,0.22627936,-0.11276553,-0.33227456,-0.11389337,-0.13998306,0.5368851,0.00679631,0.38450742,-0.58557826,0.66881263,0.22311153,-0.22616677,-0.22296157,-0.07543417,0.07733335,0.28968012,0.5961131,-0.95946825,-0.342812,0.02925641,-0.5549316,-0.6482344,0.4855427,-0.42043835,0.05686235,0.41894346,0.06049568,-0.1877669,0.02541642,0.45266458,-0.6461855,-0.3132898,0.02485753,-0.3651552,0.52045876,0.16214456,-0.38728625,0.27288225,-0.33171973,0.06296971,0.62237763,0.6716506,-1.0355843,-0.44696012,0.42053914,0.01545625,0.45557016,0.52822983,-0.3486933,-0.01491966,0.9555983,0.48198143],[0.7447447,-0.32762256,0.5134863,-0.33676022,0.35972485,-0.20534824,0.6577004,-0.28050217,-0.64164317,0.27304924,-0.585087,-0.02322447,-0.47512278,-0.21917666,0.60652983,0.2222832,-0.28430176,-0.14452356,0.30811256,-0.12964343,-0.8771353,-0.16150263,0.05472273,0.07092328,-0.8164534,0.00360211,-0.30499625,-0.5408729,-0.34669194,0.2959955,-0.5250164,0.33199853,0.11199567,-0.24158083,0.32982305,0.11386207,0.5592464,-0.24855839,-0.15399387,0.1908002,0.35076764,0.20237239,0.05828697,0.23926271,-0.0547324,-0.12461594,0.4012631,-0.495609,0.18115191,-0.12837236,-0.13155833,0.48290315,0.03236504,0.6277302,-1.2541947,-0.08659189,0.6399277,-0.6408439,-0.22450231,-0.09416515,-0.16048388,-0.7325855,-0.06730308,-0.527041,-0.13317882,0.53191763,-0.09322517,0.53015316,0.8127156,-0.2748149,0.40679806,-0.46362424,0.2061694,0.29709274,0.14856173,0.04677856,0.38998562,-0.41643137,-0.00379464,0.60117614,-0.05587348,-0.47258213,-0.33516204,0.21925947,0.8273624,-0.7600327,0.37657318,-0.6342409,0.07118069,-0.24307333,0.16821977,-0.38040882,-0.56410605,0.67240363,0.1185186,-0.16998652,-0.17063372,-0.24819084,-0.563626,0.06527213,0.28719404,-0.15683232,-0.15855914,-0.87400013,-1.4449109,-0.2573657,0.08462758,-0.21424446,-0.22519752,0.10713418,0.00413594,-0.75425065,-1.2468513,0.3956524,-0.19370331,-0.12370994,-0.07462974,0.54547805,-0.5710107,-0.10330237,-0.60396075,0.10830843,-0.19558337,-0.54909015,0.05693136,0.19150022,-0.3017483,0.37723807,-0.22334255,-0.27589604,0.1015665,-1.271519,-0.53048354,0.21996406,-0.29752883,-0.42664525,-0.03558916,0.49533153,-0.40275747,0.711707,-0.6237921,0.26225895,-0.3463492,0.2743434,0.04589299,0.26924247,0.4985742,0.18877934,-0.19123913,-0.262596,-0.14478938,0.55808663,0.01926208,0.05177161,-0.30516744,-0.19342645,0.00733987,-0.42820176,0.2171275,0.69991493,-0.6607704,-0.17478317,-0.68243897,-1.2325805,0.30875048,0.66735953,-0.6772301,0.54613036,1.4937458,0.379117,0.50097394,-0.44713995,0.7238606,-0.20047998,-0.44007653,0.10075215,-0.27898386,-0.5417786,-0.4400663,-1.0935911,-0.84334683,0.0035066,-0.05338157,0.07749689,-0.40334448,0.39526382,0.21161044,-0.13427128,-0.00673178,-0.43727562,0.18997073,-0.6576008,-0.5184936,-0.1441341,-0.00687281,-0.16084547,-0.33250073,-0.6414486,-0.43266538,-0.0397099,-0.29574722,0.17502712,-0.15143232,0.30744082,-0.64050686,-0.02275831,-0.13700922,-0.23159382,0.3687868,0.11092009,-0.59568644,-0.1449669,0.1824792,-0.3445216,0.19344752,0.4046027,-0.1689978,0.11483231,0.10610621,0.06337585,0.0940675,-0.77660924,0.6543231,-0.47522894,0.22106366,-0.26180634,0.56732553,0.28552917,0.31177145,0.53183967,0.4841147,-0.3193696,0.06880092,0.4877625,0.66864073,-0.41306293,0.20180537,-1.060507,0.4588207,0.19464971],[0.5553723,0.32886896,-0.09451423,-0.1533994,-0.02089805,-0.3406538,-0.22797078,0.5920366,0.38682652,-0.4678641,-0.05290169,0.25520292,-0.59554535,0.16150494,0.05034206,-0.17446887,-0.19871727,0.74807745,-0.18333782,-0.19420372,0.28318462,0.10150734,-0.04174314,0.39460653,-0.26265582,-0.15395153,-0.17787883,0.762563,0.04105541,0.4150712,-0.13926376,0.30397213,0.1498856,-0.5480799,0.53197503,0.08617068,-0.30477652,0.27502805,-0.03120268,0.48738888,-0.3139809,0.85729194,0.5012094,-0.8670839,-0.40444636,0.0131172,-0.30522436,-0.01056436,-0.34354988,-0.35903308,-0.03967357,-0.42732632,0.38157263,0.04608074,-0.06779155,-0.58579177,0.47223166,0.70621437,-0.19066305,0.08798633,0.26947084,0.24386002,-0.16220936,-0.2847893,0.53055817,0.6036795,0.35378784,-0.5969066,-0.40325394,-0.23489618,0.13490538,-0.40848285,-0.65880334,-0.10453531,0.21823685,-0.5802842,0.3376327,-0.4343512,-0.39598456,0.89533496,0.5668662,0.00488925,-0.45451885,-0.0993131,-0.08306096,0.47417435,0.02904264,0.97817916,-0.4858218,-0.2538239,-1.0380607,0.94746864,0.32719368,-0.462175,-0.23157924,0.17291665,-0.12637779,0.04768227,-0.56332123,0.01946979,0.6908392,0.12238771,1.1105595,-0.4488425,-0.4391849,0.43982416,-0.5224944,0.03113357,-0.49240875,-0.79546094,-0.6047064,0.53034294,0.22738886,-0.4673194,0.23488776,-0.36378542,0.34193027,0.03384606,-0.99674207,0.6460603,-0.43148047,-0.5541306,0.31757486,0.35913518,0.08222469,0.72723264,0.39358646,-0.05911311,-0.8387869,-1.4749442,0.42552564,-0.2686024,0.06279588,-0.04693916,0.29447702,0.11328612,0.7772384,-0.13841572,0.70469695,0.39231288,-1.0871278,-0.3446214,-0.03164848,0.3595218,0.6278112,-0.53955334,-0.37396494,-0.02527526,0.56651205,-0.15561903,0.11954752,0.4738741,-0.2368195,-0.44642597,-1.3928878,0.19142649,0.45341963,-0.17721954,-0.15874663,0.4862618,-0.90891767,0.37660393,0.3038779,-0.53241414,0.2967931,0.5179033,0.24249256,0.3777031,0.19501959,-0.38359985,0.35733372,-0.29112238,0.9599142,-0.34225246,-0.54445934,0.05327908,-0.42526755,-0.17205901,0.15507586,-0.61849076,0.30666456,0.16023202,0.10227318,0.597343,0.5975121,-0.22549468,0.11847257,-0.31907398,0.3236801,-0.5091418,0.21881962,-0.68913484,-0.01649912,-0.06023493,0.3237019,0.12411223,-1.5004637,0.786635,0.6939171,0.16905719,-0.3982773,0.4665448,0.51799965,-0.551955,0.41319594,-0.82564044,0.18663777,-0.01654016,-0.01280997,0.05834332,0.56401193,0.08251765,-0.27613205,-1.1414587,-0.21470277,0.84805864,0.1705617,0.13749227,0.0432887,0.30734822,-0.25704992,-0.269239,-0.10072322,0.06428703,-0.64739406,0.08057445,-0.60515887,0.22466953,-0.68365735,-0.40804803,0.23567407,-0.38533765,-0.11984661,0.22113816,0.5618605,-0.99469167,0.01259431,-0.12054624,0.6960364,-0.68278074],[0.18704076,-0.43663743,0.20484784,-0.14143018,1.2125314,-0.02270547,-0.14285633,0.5857624,-0.44002438,0.46455213,0.20399697,-0.07477634,0.28182516,0.07830299,0.02873345,-0.89860034,0.24963844,-0.45525718,-0.83110386,0.48812607,0.17815417,0.03011944,0.5846698,-0.2379402,-1.3010125,-0.31594732,-0.4755698,0.5476458,-0.8432769,0.19757906,-1.1632837,-0.36917725,0.90485024,-0.7451567,-0.39775956,0.16428545,-0.39683756,0.56666034,0.70674,0.23038478,-0.4047484,0.32257017,0.1343452,0.25099206,0.41792476,0.04349242,0.01657382,-0.28389123,0.65177655,0.03385475,0.4618625,-0.39933908,0.44971797,-0.19507821,-0.5323632,0.3969748,-0.45473194,-0.10720256,0.19712189,-0.50842243,0.4154163,-0.6222376,0.21027663,-0.6035212,0.12338241,-0.24342787,-0.48598346,0.34037766,0.30569035,-0.47374806,-0.947261,-0.5057274,-0.40018398,0.7284856,-0.04031961,0.16005412,-0.32738322,0.25823927,-0.19550633,-0.00528028,-0.22668009,-0.825238,0.62214416,-1.1118,0.6807979,-0.3140952,-0.27916357,-0.07132946,-0.52698606,0.04608862,-0.28172925,-0.13365479,-0.11170347,-0.02274193,0.47523466,-0.0119028,-0.5339517,0.24068835,-0.73775005,0.71730524,-0.46724415,-0.47641197,0.40685755,0.10451869,0.47062948,0.5939194,-0.43127465,0.10710616,0.0703769,-0.22130585,-0.33632845,0.15052013,-0.24374124,0.2056235,0.6524913,-0.16775694,-0.47806197,-0.40422076,0.12762466,0.64236176,-0.04432315,-0.23487245,-0.54550344,0.34912768,0.5074556,-0.1190443,-0.18868163,-0.2083374,0.36047417,-0.3027345,0.49240622,-0.42136002,0.15561274,0.91209126,-0.04187692,-0.5149839,-0.5251755,0.31107223,-1.0105171,0.14398482,0.93609613,0.12283298,-0.3588089,0.36157838,-0.3308531,-0.63110113,-0.19897795,-0.7219029,0.5697549,-0.37004814,0.5206662,-0.28600147,0.4133881,0.42733777,-0.9115904,-0.35480735,0.14568587,-1.5783741,0.13444579,0.53996664,-0.21937656,0.14368999,-0.11478673,-0.3115859,-0.47326714,-0.35234743,-0.34588662,-0.06613672,0.6595697,0.5998655,-0.26620743,0.09428454,0.49771315,0.02770242,0.8075503,-0.08905005,-0.06660511,-0.62861085,-0.4729476,0.6414505,-0.13222,0.07541392,0.5735089,0.7256916,0.01743825,0.07213424,0.37014714,0.03105565,0.21575727,-0.36545455,-0.43355155,0.5179919,0.2576876,-0.17709714,0.3436955,-0.43961647,0.30260167,-0.5210548,0.49252814,0.2600895,-0.17604505,-0.00369761,0.31978723,0.20238183,0.29659465,0.15541159,0.43060204,0.47418663,0.11794943,0.29886532,0.00999183,-0.5139823,0.56964236,-0.25123397,0.41617608,-0.04548756,0.20638625,-0.24049559,0.38642973,0.02051221,-0.4179654,-0.4783796,0.09542823,-0.22158617,-0.38076165,0.38059145,-0.26443923,0.3767577,-0.46105912,-0.46900234,-0.1556242,-0.00228382,0.5859132,-0.18135908,0.04046452,0.11076737,0.4298824,-0.12179305,0.3348913,0.1655319],[-0.08211599,-0.13150969,-0.5677466,0.19842412,-0.16678911,-0.36203584,0.26092753,0.5113136,-0.52650094,0.03253211,0.16000775,-0.35434237,-0.29804808,0.5353869,0.1921065,-0.744112,0.18349545,0.1081271,-0.3508056,-0.07656876,-0.3413679,-0.16642249,0.10195103,-0.7593739,-0.9944871,-0.2934623,-0.38761327,0.26974344,-0.05714814,0.36625588,-0.30557293,-0.5477499,0.31274787,-0.22491515,0.12386875,0.14971101,0.00912988,0.89567345,-1.0419545,-0.42208707,-1.1453027,0.4032437,0.42201284,0.5759263,0.27612025,-0.02071889,0.01230403,0.23278318,0.60658324,0.23059571,0.25716037,-0.31460902,0.926332,0.04084332,-0.1863899,0.33448267,-0.56211793,0.30462736,-0.0567941,-0.65128,-0.32976717,-0.6260485,-0.00866137,-0.44532663,0.87772524,-0.06238167,-0.18610379,0.09852847,-0.30678436,0.03915316,-0.00186747,0.12505679,-0.0761309,0.26438448,-0.74351746,0.8929396,0.5224594,-0.79293,0.03957914,-0.5134159,-0.25483644,0.38478696,-0.5053933,0.28349453,0.03121199,-0.10368665,-0.09506144,-0.22590004,0.4490639,0.33111107,-0.5923556,0.12535784,0.09004454,-0.26597777,-0.13328102,0.58985513,-0.36563075,0.4062224,-0.4660649,0.40500247,-0.15033467,0.2983246,0.46794552,-0.6138764,0.140491,-0.19238783,0.5772361,0.16199917,-0.48226807,0.22761692,-1.1626911,0.55760825,0.08299046,-0.13163835,0.08644544,-0.12407214,0.23137526,0.09221526,0.29288104,0.07964304,0.27177218,-0.07048588,0.35560948,-0.41409054,-0.58496153,0.49313903,0.0510082,0.1598684,0.4757268,-0.2866513,-0.09629686,-0.8118217,-0.50903744,0.67810184,0.29252222,-0.54315823,0.5519888,-0.6507982,0.2971519,0.8688004,-0.55130774,-1.1059084,-0.6845115,0.1779454,-0.12307485,0.56774765,0.35489014,-0.26441404,0.38719034,0.08055096,-0.03205176,-0.2600397,0.843799,-0.30401453,-0.2224838,0.28313196,0.19227085,-0.11271258,-0.7656128,-0.38403273,-0.52476877,0.13539524,0.38589394,-0.84436536,0.52158356,-0.06684959,0.56097525,0.06267048,0.5246469,-0.8321221,-0.32513463,0.90991694,-1.1038173,-0.05710242,0.36174667,0.15975857,0.08114043,0.24385259,0.86126786,0.5639765,0.19800742,0.05984332,0.9518647,0.54383653,-0.19091187,-0.33699664,-0.15991205,-0.15345408,0.5469096,-0.5370314,-0.31225944,0.08849045,-0.04933851,-0.0278632,0.58864266,-0.40728438,0.00163007,-0.01460694,1.0054375,0.34201986,0.27157485,-0.44122258,0.03703251,0.39395022,-0.3025578,0.13865428,0.09059109,-0.28472877,0.16193794,0.41268948,0.07329883,-0.14668609,-0.3962964,-0.33722132,-0.1423882,-0.03045673,-0.11132458,-0.8453888,-0.01057484,0.15702547,0.04806966,-0.34373248,-0.66686046,-0.6645011,0.10508461,0.7317948,-0.01899037,0.34757748,0.09807802,0.11696102,0.32198188,-0.54510707,-0.24497244,-0.6583416,0.09067018,0.77575904,-0.03675355,-0.4418591,-0.44297424,-0.3576379],[-3.63186210e-01,-2.45139152e-01,2.60266155e-01,-1.36977896e-01,-9.52978015e-01,1.82456188e-02,5.55751741e-01,-8.17053676e-01,1.27719387e-01,2.01737195e-01,2.17273727e-01,-9.24917042e-01,2.79481769e-01,1.02021061e-01,-1.35264564e-02,2.94752121e-01,1.40706062e-01,1.10979192e-01,-2.99668491e-01,5.17177209e-02,2.09119365e-01,3.35643417e-04,8.76607001e-02,1.53449431e-01,2.58843690e-01,2.92413712e-01,-3.75677824e-01,-2.40145490e-01,-3.32100719e-01,1.28862366e-01,2.78808147e-01,6.09969199e-01,3.96210611e-01,-1.15440771e-01,-1.07368730e-01,-3.31272691e-01,-1.00141025e+00,-5.40682673e-01,7.86872953e-02,-7.51983523e-01,-8.86496127e-01,-2.49631450e-01,1.25915572e-01,7.44147301e-01,-3.52200150e-01,-1.77642882e-01,-3.16691808e-02,-8.04159194e-02,-8.47817600e-01,4.08100605e-01,-7.26653337e-02,-5.30286551e-01,-1.15177929e-01,1.62233889e-01,-5.06707847e-01,2.66753286e-01,3.62619907e-01,2.23968118e-01,-1.45632505e-01,-4.86590981e-01,-2.11420789e-01,-5.83883166e-01,-8.37964341e-02,2.99873769e-01,-1.57629907e-01,-6.79963171e-01,3.81491333e-01,-5.73296070e-01,-8.00303996e-01,-3.15569907e-01,-9.12326992e-01,3.05890620e-01,1.43979445e-01,-1.09886251e-01,-2.60114580e-01,5.26106000e-01,-4.88436848e-01,7.61713862e-01,5.56153543e-02,-1.97043702e-01,-7.33084261e-01,-2.45924722e-02,-4.24280733e-01,-4.37378496e-01,-6.70444965e-01,-4.60144341e-01,-6.25470400e-01,-7.72890747e-01,1.93135552e-02,3.54870707e-02,-1.13221645e-01,1.07769862e-01,2.13937804e-01,3.01524810e-02,-7.06485212e-02,-2.34371111e-01,-1.10096306e-01,-3.59503031e-01,-5.88874698e-01,1.67854667e-01,2.74351418e-01,-7.02699423e-01,-1.17123686e-01,-1.12232275e-01,8.12534034e-01,-4.02297616e-01,1.91560715e-01,-6.47416234e-01,-8.16854000e-01,-3.37615252e-01,1.81206405e-01,2.16765344e-01,8.82229805e-01,5.00985205e-01,2.24558860e-01,1.21698841e-01,1.19312382e+00,-1.16938397e-01,-5.09548970e-02,-1.64387807e-01,5.75201437e-02,3.19274932e-01,1.24375597e-01,-1.39721483e-01,1.37244746e-01,6.13376498e-01,5.05437076e-01,-3.17109615e-01,-2.44651273e-01,-5.31211197e-01,-4.02335793e-01,7.92354718e-02,4.15014885e-02,3.89495432e-01,1.36961281e-01,5.44536486e-02,9.60149318e-02,-6.83452263e-02,-3.17564815e-01,-1.75183058e-01,-1.46723688e-01,-2.92172670e-01,3.84340510e-02,-1.93535641e-01,-6.80009961e-01,6.39230311e-01,-7.56664515e-01,-6.31679595e-01,-6.78179443e-01,-1.34103525e+00,-3.52780044e-01,-1.68505460e-02,9.00018096e-01,1.79180559e-02,-8.92587900e-01,-3.59757841e-01,8.23727906e-01,1.11058615e-01,-2.98060000e-01,6.28699005e-01,2.73484945e-01,1.01064444e+00,-1.10893376e-01,-2.37261355e-01,-2.00372059e-02,-1.80589586e-01,-1.03066333e-01,1.35262355e-01,-1.63266268e-02,4.69134711e-02,1.06999688e-01,-5.22617221e-01,9.65493917e-01,-2.48022094e-01,6.54238105e-01,4.35240090e-01,8.77916634e-01,-2.81810254e-01,5.02886415e-01,1.73770577e-01,8.21793437e-01,1.18113324e-01,-1.17961876e-01,-9.45846736e-01,1.35377407e-01,-1.37187257e-01,-1.13404119e+00,1.71289161e-01,9.21015590e-02,-4.70974028e-01,2.55721390e-01,3.03587914e-01,-1.35818735e-01,2.53086448e-01,6.58335686e-01,-7.38731921e-02,4.64426517e-01,-7.05290735e-02,5.94152033e-01,2.84866780e-01,2.51303762e-01,5.33325315e-01,5.95337264e-02,-3.47590446e-01,9.79779884e-02,-1.03263639e-01,-3.17814142e-01,9.29947644e-02,-7.20995367e-01,-4.61660653e-01,3.80386800e-01,2.24092856e-01,2.73068011e-01,-7.94991255e-02,-1.08139299e-01,4.92632724e-02,-1.59306183e-01,-2.93946922e-01,-3.42320770e-01,2.02208742e-01,-3.57800543e-01,4.03486520e-01,6.16141021e-01,1.22225654e+00,-5.53804226e-02,3.32088202e-01,1.71338439e-01,1.99973881e-01,-2.66276389e-01,-3.34810875e-02,7.52796113e-01,-7.43666366e-02,-6.92156732e-01,-2.21424535e-01,-5.82043454e-02,-1.39149916e+00,-1.30076602e-01,-9.40213621e-01,2.64061362e-01,-7.03451410e-02],[-1.57783842e+00,-1.05888978e-01,2.55045086e-01,3.89997661e-01,2.49346215e-02,-7.94819277e-03,-2.20866930e-02,2.59111132e-02,-3.04609656e-01,-4.34102505e-01,1.95398003e-01,3.37995201e-01,6.06514275e-01,3.19943726e-01,1.21542569e-02,6.54651701e-01,-2.50990957e-01,-2.50149131e-01,-4.09380198e-01,3.63398284e-01,-6.72041178e-02,5.39261177e-02,-2.99190283e-01,-5.38061440e-01,-4.58546132e-01,4.72991645e-01,-7.66852498e-01,-6.36956036e-01,-4.82407026e-02,3.24247599e-01,5.62044144e-01,-1.22107498e-01,3.09880704e-01,-1.32947278e+00,-1.01991609e-01,-5.31336308e-01,3.00726384e-01,1.07380345e-01,-1.75198644e-01,-5.23347318e-01,-6.84128702e-03,3.51810843e-01,7.63165280e-02,-4.03387547e-01,-1.11415647e-01,-2.24888876e-01,5.07410944e-01,1.54247165e-01,-7.09934950e-01,5.35637856e-01,8.61225784e-01,-3.03160981e-03,-1.64627120e-01,1.28191799e-01,-3.35429490e-01,-3.23119879e-01,5.64114988e-01,2.39189580e-01,-3.02913159e-01,-1.77504010e-02,5.37531018e-01,2.93901451e-02,5.57015002e-01,2.28350878e-01,-2.88946480e-01,-3.16643476e-01,1.53944232e-02,-7.45333612e-01,-7.48039722e-01,-2.36753225e-01,1.90349929e-02,-2.26271212e-01,-2.66600132e-01,-3.26367319e-01,-1.38135731e-01,1.40891716e-01,3.10111284e-01,2.75625020e-01,-1.45288438e-01,1.21784270e-01,2.95938607e-02,-3.50075960e-01,3.19752723e-01,3.90014678e-01,-2.97288358e-01,-5.29210746e-01,1.55988902e-01,-2.69490689e-01,2.55081862e-01,3.02971393e-01,1.14193487e+00,-5.01965247e-02,1.13721550e-01,-2.06622198e-01,2.44535610e-01,3.45658027e-02,-6.15167439e-01,7.72216260e-01,2.00860500e-01,1.29366949e-01,-8.04175436e-02,6.63014710e-01,1.78871915e-01,-6.33850217e-01,3.60917673e-02,2.99884498e-01,1.36122063e-01,2.18948439e-01,-2.40693748e-01,-8.34198117e-01,-5.39850332e-02,-2.93554515e-01,2.62795925e-01,-5.75191602e-02,-1.74391136e-01,-2.87864119e-01,-5.02198458e-01,2.20201313e-01,-6.98035181e-01,4.77736801e-01,-1.02017748e+00,-1.66502774e-01,2.69567251e-01,1.14231491e+00,-1.98461330e+00,-4.88871574e-01,-9.98153508e-01,2.18570784e-01,3.25189270e-02,-2.81777352e-01,-5.64545214e-01,5.97975731e-01,-5.74194312e-01,-3.05679321e-01,-2.59834290e-01,2.70035654e-01,-2.87369162e-01,8.44283476e-02,5.62144458e-01,-2.27710202e-01,-8.88041258e-02,5.70793808e-01,2.77787387e-01,-8.14519450e-03,6.15403056e-01,2.16372848e-01,-2.31709972e-01,2.02573780e-02,-3.34368199e-01,-2.03038901e-02,-3.80973011e-01,3.83509517e-01,7.40118563e-01,-1.64651796e-01,2.40387991e-01,2.60074198e-01,-5.66334166e-02,-6.36831701e-01,-2.08745524e-01,5.97279929e-02,-2.23265618e-01,2.55020410e-01,3.46880138e-01,-3.34396124e-01,4.17011797e-01,1.62696093e-01,-3.00043941e-01,-1.79851355e-04,-1.85434312e-01,6.14384525e-02,8.70032668e-01,1.13133654e-01,-1.71962619e-01,-1.17133725e+00,-1.66018695e-01,-3.00596923e-01,-7.10466564e-01,3.64612103e-01,3.99357170e-01,2.30465814e-01,-2.21035201e-02,-6.58556283e-01,-1.51699126e-01,4.64683324e-01,-8.88692797e-04,-4.11691695e-01,-1.50572225e-01,4.34944242e-01,6.73817396e-01,-2.84794778e-01,5.36269993e-02,-1.75213069e-01,-4.04413819e-01,2.54135311e-01,-1.48140788e-01,-5.12545288e-01,5.85772991e-01,-2.20836803e-01,-3.95527435e-03,-1.34229466e-01,4.50249374e-01,-1.45229980e-01,-5.49804151e-01,5.37563860e-01,-6.04896486e-01,-2.67637640e-01,-5.28055727e-01,-5.69678238e-03,3.40075314e-01,-5.31211257e-01,-9.01541471e-01,2.90926307e-01,-6.20035157e-02,-6.26740903e-02,5.22583425e-01,-1.84545800e-01,-3.74051243e-01,3.70044351e-01,-9.26427245e-01,-6.56121552e-01,1.00525320e+00,-8.64914179e-01,7.70420313e-01,-2.68008620e-01,-7.87397325e-01,7.86318779e-01,1.42679289e-01,-2.64542878e-01,-2.91760147e-01,-2.27817744e-01,-2.54715094e-03,-1.35426641e-01,1.84923604e-01,6.21826410e-01,3.02118331e-01,2.64683157e-01,-8.76143202e-02,-5.15385151e-01,-1.46773815e-01,-7.21381485e-01],[-0.09896919,-0.45468712,0.6940633,0.47064784,-0.31582215,-0.19131577,-1.0109003,0.48630774,-0.52674794,0.80194134,0.85955364,0.10767168,0.42340145,-0.96439534,-0.4119542,0.12456081,0.14277363,-0.08858114,-0.18953489,-0.00892096,-0.1899604,-0.11794236,0.42227563,0.54984874,-1.0881752,-0.46123263,-0.200561,0.14233176,0.25832814,0.48361707,0.3376364,0.54751146,0.01490229,-0.84572196,0.3517654,-0.5640535,-0.0884342,-0.1525372,-0.26352605,0.69449437,-0.09100062,0.43568492,0.03376143,-0.4331493,0.22465728,0.01218238,-0.18586297,0.06176165,-0.23830341,-0.01305732,-0.8797596,0.3076654,-1.0315244,0.85760444,0.7364916,0.35644436,-0.24178788,0.0672951,-0.1798337,-0.46761447,0.28387994,0.5246948,0.11117986,0.01700314,-0.24181049,0.43256542,-0.07175341,0.00351273,-0.10167033,0.09920874,-0.3948112,0.2707696,-0.01052655,-0.4885872,-0.14745325,-0.40122452,0.10462613,0.12191084,-0.375247,0.39645684,-0.5005076,0.35488346,-0.19436218,0.19130245,0.29150927,-0.6153683,-0.24608354,-0.08238316,0.22554968,-0.8354288,-0.46784446,-0.24660042,0.37853074,-0.05472173,0.48114377,-0.28541833,-0.5362204,-0.87181014,-0.406898,-0.17226484,0.11784711,-0.2936488,0.9622487,-0.05343453,-0.15091957,0.17422758,-0.4537735,-0.5016123,-0.35352293,-0.20779398,0.22239387,0.32858053,-0.39048153,0.74787855,0.0630094,0.02565299,-0.25615427,-0.46363163,0.23174714,0.07491291,0.4827317,0.23617189,0.33394378,0.03798236,0.30651313,0.30605638,-0.34863093,0.12558909,0.06683244,0.0402307,0.4453201,0.7595486,0.06825137,-0.8996666,-0.3360582,0.45763403,0.06913994,-0.26804328,0.3230115,-0.9193936,-0.38105342,0.35135236,-0.47572154,-0.09091049,-0.45289344,-0.12092122,0.21350738,-0.334814,0.42172936,-1.3176668,0.16581184,-0.718333,-0.3229581,-0.05577612,0.61663073,-0.608496,-0.10071468,-0.22869872,0.5368745,0.02099286,0.7464759,-0.09277563,-0.9815727,0.5193544,0.8538374,0.00504753,-0.0626943,0.12910375,-1.0055392,-0.65164095,-0.31059438,-0.6611884,0.54501283,0.603959,0.36403435,0.04221954,-0.27544647,0.23332122,0.17354274,-0.7422716,0.4593678,0.22793609,-0.6441671,-0.21479763,0.598223,0.39879864,-0.65128267,0.42640278,0.39227864,-0.21253404,-0.8217679,-0.3404491,0.20773655,-0.13497362,0.19786257,-0.09537543,-0.30847463,0.62742656,-0.212878,-0.6682293,-0.0997787,0.2342434,-0.15176572,-0.24371006,0.44869936,-0.5967203,-0.53196317,-0.23593508,-0.20366755,0.25272027,-0.46176365,-0.15025185,-0.21113734,-0.0629633,-0.2734345,0.10330448,-0.07000168,-0.46683693,0.41840264,0.3640354,0.2896087,0.7142519,0.008476,0.12757091,-0.23292507,-0.5141777,-0.07959836,-0.59559023,0.45065463,0.6299028,0.00595649,-0.02066405,-0.21232666,0.25057966,-0.50546455,-0.39579368,0.13960509,-1.3024427,0.5751679,-0.8645247],[-0.04091727,0.30938858,0.0079092,0.35123485,-0.0163823,0.06688592,0.8115877,0.24801429,0.06113019,0.03602256,0.11673225,0.0892402,-0.28440017,-0.49281698,0.28228086,0.37946227,0.36153314,0.32277587,0.6783551,0.5098301,0.2517059,-0.92172045,-0.45942885,-0.26421353,-0.48964983,0.18957137,-0.15378454,-0.2887887,0.3289443,-0.60038114,0.25260004,-0.18336783,-0.22453171,-0.28881893,-0.3344353,0.18394831,0.27469307,-0.4158906,0.55883133,-0.6489369,-0.15313642,0.646356,-0.48372847,0.0938338,-0.00258408,0.40289378,0.13277726,-0.3732914,-0.21841764,0.56309843,-1.136124,-0.46663252,0.64646,-0.01035009,0.07749652,0.18339473,0.677491,0.26343936,-0.07142202,-0.36848843,0.7582569,-0.74723643,0.34828144,0.07520781,-0.24115486,-0.1377118,0.9051843,-0.14946802,0.2548088,-0.10838038,0.24911192,-0.25852394,-0.02203699,0.19087158,0.32732207,-0.20169842,-0.6802527,-0.5637464,0.27715105,-0.20612164,-0.30119157,0.9045355,0.04400118,-0.01770601,-0.77387965,-0.97730535,-0.10424587,-0.2747641,-0.1963335,0.5104368,0.44803393,0.03209129,0.37374204,-0.7391978,0.5983496,0.6351645,-0.18337858,-0.01882317,-0.93752027,0.03046464,0.12680379,-0.35670772,-0.32929528,0.5874238,0.09722929,-0.20600526,-0.3023668,-0.37283382,-0.59676486,-0.2941199,0.23358184,0.31827736,-0.5968421,0.44610485,0.52137095,0.5372313,-0.3181518,-0.37106255,0.19285221,-0.13593061,0.4261563,0.02819289,0.33897,0.05992291,0.43904102,0.2977766,0.63122964,0.32653856,-0.5457157,0.11865975,-0.9492269,-0.938762,0.00447434,-0.10515653,-0.22815448,-0.02377167,-0.5552141,-0.09154695,-0.5429969,-0.6498989,0.13151656,-0.03595448,-0.21633688,0.05037902,-0.21290536,-0.32594556,0.23521362,-0.01252714,0.3731504,0.22476742,0.39645812,0.49048728,-0.34842348,-0.14108986,-0.5863025,0.23054579,-0.2988811,-0.38946682,0.0805876,-0.97018474,0.5163949,0.5776036,0.15096772,-0.06892468,-0.15068902,0.20357671,-0.14602362,-0.36670578,-0.6485284,-0.9147407,0.5449155,-0.18179819,-0.25131035,-0.41998285,-0.90518916,-0.1588087,-0.10559078,-0.64552337,-0.20817894,0.26491645,-0.00604947,0.16818705,0.56529105,0.50670224,-0.5897295,0.36642003,-0.16001299,0.08325069,-0.22719397,-0.13675281,-1.0739912,-0.13142005,-1.164795,-1.3800986,0.18294924,0.01407296,0.49024498,-0.35494748,-0.4209029,0.3247906,-0.5502168,-0.671631,-0.15613389,0.00363035,-0.43491897,-0.16480297,0.25851205,0.18633786,-0.63566864,0.09368404,-0.24423032,0.37548402,0.24174921,-0.5890373,0.3788827,0.14002109,0.7521408,-0.30894285,0.19771457,-0.7984421,0.19954637,-0.06943695,0.41550884,-0.3494596,-0.05499307,-0.48552978,0.10076665,0.31874332,-0.11993225,0.09376404,0.2637372,0.53098613,-0.27361557,-0.6895248,0.01864156,0.03543179,0.77288496,0.91094416,-0.9544751,-0.76624036],[0.28113636,-0.13643038,0.5001623,0.2008608,0.01498792,0.42916203,-0.16262501,-0.3364545,-0.72655004,0.38789266,0.21356021,-0.3954817,0.26323208,-0.49616587,-0.23982482,-0.19138741,0.10464725,-0.04891093,0.07598024,-0.6230152,0.23923388,-0.3823695,-0.11643462,-0.57921696,-0.08516719,-0.24765027,-0.30367813,-0.5121605,-0.05885382,0.20676774,-0.23060757,-0.40636498,-0.22512019,0.45557013,0.0504224,0.17056559,0.11693612,-0.6337236,0.45646334,-0.02521288,-0.63188434,0.0935945,-0.13437103,0.06081279,-0.43214422,0.17071,0.2198938,0.13431074,0.24688733,-0.09374033,0.19816293,0.18821083,0.14579362,-0.488538,-0.35734802,0.8681639,-0.16923822,0.19804333,0.09173082,-0.47703367,0.6252496,-0.12527087,0.01380174,-0.24979897,-0.4307182,-0.3229594,0.03449519,-0.7741666,-0.48199743,0.7159889,-0.26800653,-0.9087342,-0.01574209,-0.58747375,-0.11248821,-0.31773865,-0.06751109,-0.85818344,-0.08468807,0.00283963,-0.7799292,0.14452018,0.703257,-0.00627808,-0.33159873,-0.29501605,-0.38634175,0.20594479,-0.2767044,-0.42282572,-0.38448817,-0.36855245,-0.49098825,-0.5996224,-0.17667161,0.09620372,0.6116165,-0.13074969,-0.29794374,0.4787314,0.2686383,0.23157194,-1.7278506,0.32557702,-0.5568939,-0.09594048,-0.3478698,0.77474487,-0.88933015,-0.6231469,0.1636104,0.26486388,-0.2539568,-0.14639342,-0.11770231,-0.2649679,-0.44938722,0.43152645,0.3576845,-0.0231597,0.04194924,-0.13257378,-0.3594059,-0.79541755,-0.23701377,-0.8028582,0.5409405,0.08499935,-1.0454161,0.05588878,0.17821902,0.4959072,0.33847192,-1.008372,0.44103456,-0.09340055,0.04328639,0.0977385,0.16440943,-0.72481716,0.45279738,-0.4197125,-0.16071956,-0.15029944,0.49868983,-0.81913674,-0.834262,-0.49529773,-0.2652222,0.57538354,-0.15837294,0.05630504,-0.22360441,0.6105152,0.09668256,0.47199136,-0.228116,0.4343121,-0.3023023,0.95688915,-1.0750529,-0.5952271,-0.28664577,-0.11656183,-0.75847894,0.43776792,-0.25934273,-1.3628391,-0.13621399,0.77860624,-0.13072751,-0.59021795,-0.6073125,0.03439995,0.2806287,-0.49120802,-0.278568,0.8948305,0.00494965,-0.2130083,-0.2450536,0.17039467,-0.49954954,0.49500868,-0.2817187,-0.04842123,0.5806515,-0.15413244,-0.12389589,0.3079781,0.7089956,0.78637344,0.52148587,0.95204055,0.14168721,-0.91978896,-0.4000858,-0.00264999,-0.5426064,-0.71605664,-0.35145175,0.03586662,0.22959219,0.8412147,-0.43634894,-0.23775594,0.33840612,-0.38311774,0.2886301,0.80936015,-1.1472198,0.19491471,0.25037766,0.17552555,-0.2740386,0.19476043,0.5904705,-0.16031861,0.4610098,-0.38394755,0.03816318,-0.40209514,-0.05010071,0.01048978,-0.51123387,0.3375014,0.16542189,0.14874011,-0.21890846,-0.12029174,0.18466209,0.8106264,0.7234434,-0.49973795,-0.38575515,0.55287004,0.2745209,0.17971265,0.29117087,-0.07012514],[-0.7920101,-0.45793372,-0.03296553,0.04218375,-0.53299856,0.16085559,0.5185007,0.07533625,0.45873937,-0.36515257,0.6658093,0.6378742,0.45769656,-0.5722902,-0.2666013,0.32301137,0.42281446,0.12377624,-0.24440664,0.10266905,-0.67926586,0.08634803,0.93626785,-0.6069665,0.04798643,-0.43863565,0.2819958,0.36466178,-0.33651856,0.24656981,0.385342,0.24669424,0.16245265,0.47876406,-0.07033098,-0.19762993,-0.18130621,-0.6591406,0.26917794,0.15223,-0.17380844,-0.7116864,-0.17010795,-0.24428006,0.15747717,0.01254618,0.24592116,-0.4330301,0.1356552,0.02773713,-0.38751352,-0.06282153,-0.4289622,0.18665385,0.76343936,0.6242718,0.24049768,-0.55864084,-0.1487491,0.10124578,-0.32729772,1.0931128,-0.46195313,0.44893378,-0.54516494,-0.2635472,-0.361186,0.5191222,-0.6304184,0.0537326,0.04858889,-0.7118578,-0.12572975,-0.9610857,-0.23589328,0.43444,-1.2402785,-0.4920137,-0.08968247,-0.21694174,-0.47550195,0.10825628,0.79816896,0.14339381,0.2456544,-0.37064052,-0.82351327,0.0619166,-0.18482399,0.35541987,-0.08429322,-0.850776,0.24959448,-0.4965422,0.05710496,-0.2719719,0.58008677,-0.91963434,0.44308296,-0.27636144,-0.49539822,0.0328376,-0.6839463,-0.17923442,0.41264665,-0.04758951,0.10625404,1.0373503,-0.23313469,0.4363748,-0.26837206,0.23468769,0.07665941,-0.2601843,-0.23386386,-0.04655405,0.06847639,-0.2185297,0.94158185,0.0452167,-0.42445958,0.6107045,0.3388521,-0.7842631,0.01778915,0.19598883,-0.39983198,0.17776428,-0.1415276,-0.13467216,-0.02270298,0.34128258,0.9023825,0.05334586,-0.25711665,0.03656672,0.08552979,0.4202972,0.16492613,0.11678772,0.1408113,0.8899436,0.03176535,-0.83072376,0.5098909,-0.13912128,0.27203757,0.4739395,-0.01894046,-0.30511853,-0.01610642,-0.18084343,-0.06726953,-0.64691734,0.22082418,0.0095176,0.4152103,-0.00257272,0.6282469,0.6217179,-0.26921317,0.04449381,0.00448805,0.310336,0.20629084,-1.2965806,-0.11440033,0.3842041,0.02671504,0.08517751,-0.36124238,-0.03654218,-0.7882861,0.24923864,0.3657732,-0.43579423,0.3331763,0.15633054,0.0787022,-0.00461484,-0.2484827,-0.5303991,-0.43182892,0.5647703,0.19769016,0.24933146,0.23428832,-0.24086626,-0.38557658,-0.02038566,0.6659762,-1.372816,-1.1027302,0.30333716,0.01517128,0.2543945,0.43739182,-0.7743419,-1.6199745,0.24832024,0.2836827,-0.20478114,-0.87267846,0.11916374,-0.465672,0.15638694,0.04818954,0.05472913,-0.00681544,-0.05432321,-0.24262,0.03745871,0.3884116,0.11542489,-0.3244661,-0.3209372,-0.5879023,-1.1045997,-0.5407416,0.1862396,-0.1592111,0.8649093,0.4299428,-0.10170837,-0.01614991,-0.7320426,-0.6333455,-0.6364176,-0.04831939,0.05423636,0.23490599,0.57493275,-0.68393356,-0.729471,0.4437174,-0.2818794,-0.31208423,-1.0800457,-0.3656899,0.17797504],[-0.7034201,-0.27344757,0.73816085,0.40039784,0.10413647,0.02894333,0.27403545,0.25444663,0.30931863,-0.53170294,0.06112491,0.53871346,0.54775083,0.12250853,0.43131927,0.08109816,-0.07260156,0.5007249,-0.15309314,0.03596688,-0.08993507,0.5747096,-1.0893137,-0.16255583,-0.1136484,-0.677777,-0.3156351,0.13242403,0.6910628,0.25846496,-0.34533748,0.07264084,0.341838,-0.03378496,0.02022774,0.2637772,-0.4847956,-0.66902673,-0.51893556,-0.46329486,0.6687994,-0.29220057,-0.6244077,-0.13689935,0.06920283,-0.03297639,-0.15879554,0.0243539,0.0271744,0.16970758,-0.17474225,-0.8944001,0.4354958,0.50634545,-0.38253695,-0.37384754,-0.31806737,-0.15523,0.04617351,-0.09753881,-0.95899135,-0.6191649,0.20178078,-0.1279125,0.7195645,0.62887084,-0.8244409,-0.12009459,-0.36632696,-0.8020873,0.14841819,-0.43311417,0.17821148,0.32532552,-0.51174074,0.04919428,-0.3347315,-0.4331328,0.06151306,-0.05134131,0.1876662,-0.43456438,-0.2983407,0.00498271,0.16460948,0.39477962,-0.20245619,0.1275614,-0.35434937,0.5308137,-0.01825922,-0.2933963,0.34141174,-0.2581971,0.43081868,0.35722908,0.43888947,-0.1285741,0.44750643,0.0972881,-0.00331448,-0.1783234,-0.02690968,-0.72939867,0.60252875,-0.41290644,-0.11946571,0.0146787,-0.25598162,0.7068013,0.43688452,0.25000405,-0.69169766,-0.17461516,-0.13952813,0.54220057,0.14750057,-0.06395845,-0.8157038,-0.685718,-0.7981471,0.2037795,-0.09378776,0.23575032,-0.9258512,-0.32006738,-0.12805083,0.45935076,0.13848117,-0.47492212,0.3611125,-0.27556145,-0.31794232,-0.23272529,0.09996991,0.3801431,-0.3279091,0.24121907,-0.4980603,0.43771031,-0.424924,0.65519845,0.05982247,0.49316147,0.40971056,-0.08656552,0.46344715,-0.8831246,0.27980942,-0.19462547,0.14117503,0.3338963,0.83221406,-0.5436671,0.34519905,-0.28300187,-0.950411,-0.48222506,-0.86035407,0.11075709,1.0625527,-0.25576198,-0.62290484,0.8646652,0.07445414,-0.4584317,-0.5199677,-0.54680043,-0.5784917,-0.7924811,1.0767369,-0.8926223,0.1272392,-0.09346544,0.33239642,0.16402848,-0.5797804,0.15438764,0.40892044,-0.4212219,0.45573756,-0.00177504,-0.24717075,0.5993818,0.2019121,0.44236988,0.1444815,0.20343812,-0.1033484,-0.69363356,-0.00573796,-0.5265855,-0.5532391,0.3659845,-0.5793671,0.43846858,-0.4189313,-0.1730922,0.68538684,0.12326387,-0.91523445,0.50421166,0.20952192,-0.02554428,-0.26582828,0.3316816,-0.11079884,0.3704602,0.17224614,-0.10981428,0.24147345,-0.7385836,-0.36714563,-0.02383552,-0.4191703,-0.18891372,-0.4626994,-0.39201626,-0.30783975,0.14807944,0.33225372,-0.5052993,0.35556003,0.36704275,-0.83951604,-0.21043281,-0.2953302,-0.6313715,-0.06846033,-0.1048775,0.05339058,0.5706118,-0.47822034,-0.0918541,-0.4651748,0.27879095,-0.40141544,0.03051556,-0.17156465,-0.35051185],[0.43254423,-0.92976004,0.13131294,0.39729452,-0.44928458,0.21658841,0.6488737,-0.545167,0.16022822,-0.69503915,0.29084843,0.06444948,-0.16203617,0.11085527,0.39196634,0.22463305,-0.54782206,0.4270228,0.4466226,0.06928002,0.47134218,0.15314856,-0.44804755,-0.17643395,0.03083243,0.08535074,0.7955088,0.3298069,-0.38164398,0.11105707,0.24690907,-0.7238905,0.11104696,0.7364471,-0.312013,-0.4115752,0.31507576,0.6092892,0.07068498,0.3134276,-0.04882323,-0.0281273,-1.0269177,-0.06504598,0.54609334,-0.09159525,-0.6358853,-0.38662523,-0.28154856,-0.272133,-0.00540229,-0.4381997,-0.29721206,-0.6396641,0.02005085,-0.1966051,-0.21466757,0.5414612,-0.32560664,-0.63255465,-0.17911066,0.0640179,-0.21541668,0.11838303,-0.32656085,0.01049686,-0.59222966,0.30144617,0.5661356,0.05368148,-1.5872766,-0.5552764,-0.41400334,-0.27454582,0.45576873,-0.36372834,-0.56666565,-1.1478125,0.42141768,-0.09507532,0.11325992,-0.35195768,0.08063848,0.09228329,-0.31225148,-0.5661044,-0.44971424,-0.05706544,-0.15004708,0.8546889,0.31777024,-0.7494812,0.11118244,-0.16443013,0.21740666,0.12946075,-0.26565152,0.4944535,-0.45728046,0.21982974,-0.13274118,-0.18231684,-0.5598006,-0.76921695,0.09419995,0.2343831,0.22489367,0.2634649,-0.9664799,0.259535,0.23940536,0.35969353,-0.4948565,0.17159009,-0.36537695,0.2959345,-0.01401463,-0.14271241,0.20580474,0.00998272,0.82134753,-0.16359802,0.14213051,0.22225517,-0.6668922,0.17524321,0.27559027,0.3384869,0.40937728,0.24298593,0.08929268,-0.28786436,-0.4247771,-0.29136345,0.01482784,-0.09994406,-0.01455174,0.10216692,0.21488181,0.34247333,0.26011756,0.10291971,-0.16432893,-0.1323969,-0.5589526,-0.5530485,0.39517385,0.04701768,-0.26985398,0.07662936,-0.8117668,0.10310499,-0.15890738,-0.26974073,0.46200964,0.02064994,-0.12949756,0.49484712,-0.30804983,-0.56686336,0.19648816,0.6455418,0.4260883,0.53898364,-0.3081734,-0.01011609,0.5966225,-0.5962405,0.17710067,-0.16128711,0.8715892,0.59169734,0.46349075,-0.32218024,0.08233168,0.02583533,-0.16552563,-0.34568587,-0.01697884,-0.70005816,-0.4481542,0.28911257,-0.37195447,0.5900556,0.55592495,0.38358516,-0.2345616,-0.13805634,-0.19902286,-0.12323618,-0.0926653,-0.12530583,-0.41229737,-0.8687761,-1.2951669,-0.09835258,0.30508757,-1.0714625,-0.09219231,0.09132989,-0.44713452,-0.57665724,0.65805966,0.76384616,-0.69259006,-0.01900995,-0.06546619,0.14104246,0.00249021,0.45849293,-0.6588285,0.23374756,-0.79113454,0.5013452,-0.03130475,-1.5049752,-0.75887525,-0.99851793,-0.26074326,0.43043086,0.4968003,0.2447731,-1.1560531,0.5181608,-0.22602473,0.0187415,0.6402625,0.11372926,0.03770681,0.5778801,-1.8679764,-0.11534235,0.39692307,-0.16569343,-0.27056465,0.24058387,-1.0322638,0.396918,0.54995704,0.6149203],[0.01465807,-0.02780825,-0.31163424,0.45385873,0.94475687,-0.6818076,-1.4949622,0.060335,-0.16580707,0.16796952,-0.7787884,-0.5075506,-0.30178,0.5556394,-0.1090141,-0.5334289,-0.49412158,-0.37213257,-0.04341728,0.21416938,-0.12486219,0.11954382,-0.43771842,0.2109244,0.01621892,0.11632709,-0.42734325,-0.0236265,-0.04102197,0.52778774,-0.05072178,-0.03271119,-0.1804716,0.5740746,0.07169069,-0.11701277,-0.11641321,-0.9572105,-0.93820643,0.37624002,0.12051652,-0.2101455,-0.14903069,1.0176121,0.5449574,-0.00819545,0.06606499,-0.8910082,-0.12600234,-0.00931824,0.24939725,-0.2130899,-0.04707469,-0.45537218,-0.7323733,-0.00977678,0.06532437,-0.38201267,0.568656,0.23554331,-0.2121025,0.8326794,0.50507826,-0.20873943,-0.39627665,-0.1724503,-0.08556238,0.1182535,0.32065567,0.08247595,-0.17628422,-0.41840127,0.2964011,0.00379047,-0.47324067,-0.23483874,0.41541928,0.5168061,0.31799176,-0.40220037,0.11741002,0.47208115,0.7026568,0.35272002,-0.6789441,0.6185695,-0.57599515,-0.04043756,-0.15263745,0.63882554,-0.59096354,-0.33591595,0.23204374,0.41894963,-0.9373921,0.2223057,-0.30286342,-0.9634219,-0.39772853,-0.46977088,-0.22132622,0.3676812,-0.32930258,-0.07272792,0.3716952,-0.33539474,0.4156305,-0.69829404,-0.5506481,0.15211403,0.21890765,0.88112444,-0.3976978,-0.487356,0.14330147,-0.43613628,0.4538211,-0.2962613,-0.81472665,-0.4798592,-0.03160538,-0.24790688,0.08872778,-0.2691485,-0.29664543,-0.38599148,0.5848477,-0.60137457,-0.528569,0.47790197,-0.6192836,0.00650395,0.07277068,0.05479458,0.55653536,0.19298953,-0.38687974,-0.2273776,-0.514165,0.22474666,0.57292837,0.14678812,-0.16265923,0.24360621,0.7789417,0.7895109,0.09093589,0.33462828,0.12392318,-0.8947842,0.17358051,0.22988166,0.12410481,-0.05996777,0.0395369,-0.15887293,-0.26997665,0.35970414,0.14505377,-0.21987021,-0.9357621,-0.41550195,-0.5529592,0.2702044,0.12733583,0.3135791,0.1774108,0.21819837,-0.17273556,0.25497213,-0.25579563,0.32784846,-0.53893286,-0.65161765,-0.33853662,0.34947813,0.506853,0.17813702,0.00266896,0.58501947,-0.5948463,0.15402797,0.3307519,-0.21485732,-0.3091067,0.15920281,0.55332756,0.3128474,0.08405946,-0.13783385,0.18751256,-0.95866644,-0.8003083,0.31412432,0.69724506,-0.29167688,0.03546161,-0.7806715,-0.46312714,0.29094464,0.38275835,-0.15134671,0.1075752,-0.22888628,0.0292599,-0.54353666,0.68549204,-0.21982121,-0.82998574,0.7269738,-0.24309693,0.01157195,0.01299037,0.03601988,0.507674,0.7510938,0.321444,0.16980158,-0.6965247,-0.30800042,-0.3412615,0.72783583,0.32337007,-0.22285384,0.07894345,0.53186864,-0.16419588,0.30298615,0.5623688,0.11530458,-0.35567123,0.04824733,0.40976918,-0.5258826,-0.71050406,0.618945,-0.7289145,0.05910775,0.0529407,0.10778868],[0.09448018,0.14426889,0.71643126,0.19129214,-0.16822101,0.27053648,-0.363292,0.16399448,-0.10765204,-0.33095405,-0.29235357,-0.28455007,-0.12795325,0.84167653,0.33480242,-0.14498252,0.27480552,-0.32825238,0.19502561,-0.3594995,0.1573245,0.01625695,-0.40449595,-0.10128607,0.09682041,0.4080286,0.02412793,-0.5500308,1.040745,0.6751472,-0.37294233,-0.26453498,-0.5454484,-0.6416165,0.00508254,-0.01043302,0.17199522,-0.65599823,-0.3488415,-0.28941926,0.22911492,0.17487192,0.31322777,0.338431,-0.8587603,0.586991,-0.29275748,0.21677051,-0.29042143,0.10264542,0.5487069,0.34891555,0.1113104,0.18437833,-0.35276324,0.3210698,-0.09054418,0.87745243,-0.36696416,-0.50059545,0.173784,-0.6871476,-0.30435768,0.39108264,0.17079413,-0.4743548,-0.47979778,0.3597257,-1.0067995,0.8884106,-0.51163876,0.06435209,-0.02867873,0.14949708,0.5631121,-0.47486985,-0.5344246,0.02989349,-0.01929599,0.6169263,0.20251808,-0.00313978,0.10402963,-0.73677534,-0.38269207,0.32988092,0.24422026,-0.35002014,-0.05763351,0.37869534,-0.3223374,0.8692023,0.12800187,0.0670365,-0.3621675,-0.74523246,-0.8988607,-0.09538655,0.16968782,0.16144526,0.56458163,0.14050671,-0.05344246,-0.12094475,-0.10918463,0.08128291,-0.00745162,0.29815063,0.00452905,-0.31896317,0.1777387,-0.30328083,-0.786389,-0.6982553,-0.59021246,0.87404937,0.20747824,-0.6953812,0.91103417,0.73621124,-0.7976652,-0.24173531,0.5240863,-0.7309887,-0.37140024,0.75191194,0.07926284,-0.48363274,-0.43639147,-0.10443553,-0.673269,0.21046227,0.05066021,-0.15418604,-0.04730123,0.41808861,0.3712311,0.09113698,0.6443276,-1.2549801,-0.5168585,0.23789102,-0.36779773,0.29989943,-0.20162947,0.28064504,-0.12346472,0.2532525,-0.04209781,0.30818126,-0.3488711,-0.47692415,-0.13050579,-0.31810716,0.6198135,-0.28813228,-0.27349418,0.8194717,-0.39645723,-0.12899514,0.5508001,0.6559851,0.90677553,-0.5225169,0.5751385,-0.37815276,-0.10701927,-0.17376697,-0.05326859,-0.41915733,-0.0436448,0.34825617,0.9628739,-0.07935944,-0.17289768,-0.7128325,-0.7639372,-0.57730246,-0.47546032,-0.7149722,-0.6273448,0.43886387,0.41852495,0.14865981,0.1761701,-0.50072813,-0.26683706,-0.16574784,-0.26839986,-0.05658861,-0.635409,0.17804013,0.16195516,0.0826425,-0.28320888,0.16911815,-0.25842312,-0.381375,-0.5501365,0.6168138,0.19271234,0.06033167,0.1649893,0.494282,-0.47022778,0.7060532,-0.3638412,-0.4557995,-0.3057302,-0.11060646,-0.18058005,0.32970065,-0.8289599,0.2767003,0.26011556,-0.79061806,0.7001734,-0.41364095,-0.16812155,0.20159905,0.3106301,-0.6093862,0.16586539,-0.24907951,0.6184837,0.17212301,-0.12729828,-0.05303308,-0.3655381,-0.20711052,0.79560065,-0.27911004,-0.5601913,-0.50874275,0.59432507,-0.4238859,-0.35184345,-0.12468547,0.51088953,-0.08048305],[8.17488600e-03,-6.99758649e-01,8.63087624e-02,1.56214252e-01,-4.07488078e-01,2.59797961e-01,-1.82924405e-01,-2.18378842e-01,-1.66660562e-01,-7.87437201e-01,6.43061399e-01,-2.07506388e-01,1.41924337e-01,-2.87886918e-01,1.61575601e-01,-9.92083788e-01,2.48532712e-01,-5.61064035e-02,4.28538807e-02,2.25740429e-02,4.82198954e-01,-5.86209655e-01,-4.78142887e-01,-2.88749039e-01,-3.69532913e-01,-8.71497989e-02,-8.88555825e-01,8.35255742e-01,6.89162672e-01,-1.07169223e+00,-4.93498594e-01,1.87934816e-01,5.77611923e-02,-2.20412865e-01,1.35086372e-01,5.82305389e-03,-5.68793595e-01,2.49856740e-01,7.56290138e-01,-1.18133135e-01,-1.73938334e-01,2.80574948e-01,-1.47367746e-01,-3.72746527e-01,2.99537390e-01,3.21387619e-01,-4.85847563e-01,-3.43992561e-01,-3.93391550e-01,9.72097516e-02,-7.34687924e-01,-2.84416139e-01,4.17041540e-01,-4.94709462e-01,1.25487670e-01,2.42713422e-01,8.93645644e-01,-3.95228207e-01,-3.00555766e-01,-1.90615639e-01,-7.22060680e-01,2.46951878e-01,1.17244413e-02,1.22727871e-01,6.33708775e-01,1.00087988e+00,-5.72978318e-01,3.59001756e-01,-3.57083797e-01,5.37591398e-01,-6.73095524e-01,8.78080726e-02,6.41307756e-02,-2.23360091e-01,1.54311538e-01,-9.67741162e-02,2.12716743e-01,8.29711974e-01,-2.67378777e-01,8.06817293e-01,3.95184666e-01,3.60231221e-01,5.24005413e-01,-4.15619522e-01,4.42251921e-01,4.76952225e-01,4.70435284e-02,4.01892662e-01,1.87271163e-01,-1.55096874e-01,-5.31683683e-01,3.14126104e-01,1.41214266e-01,-2.38520607e-01,2.13854656e-01,9.76263657e-02,5.09016871e-01,-2.55653173e-01,5.84029317e-01,2.21689597e-01,3.00151676e-01,6.25604212e-01,3.50674391e-01,-4.20503527e-01,-6.33614659e-01,-5.81843853e-01,1.53097376e-01,-7.02564061e-01,-8.71358931e-01,8.85684863e-02,1.06631406e-01,1.58318967e-01,-4.92011487e-01,2.91101411e-02,-3.03955972e-01,-1.72710240e-01,-1.32821545e-01,3.82552624e-01,6.01297319e-01,2.24915624e-01,1.92938298e-01,-1.53291613e-01,-6.40468717e-01,4.76694733e-01,7.51121283e-01,-1.48534894e-01,-9.44719315e-02,-8.48280013e-01,5.59279382e-01,5.11852980e-01,-2.35581428e-01,-4.05985385e-01,2.35920668e-01,-3.21983732e-02,-3.92335832e-01,3.72239470e-01,-1.15347219e+00,3.04806262e-01,-2.48230293e-01,-1.32981567e-02,-3.29109490e-01,2.64614046e-01,-3.29735696e-01,5.69552183e-01,-2.25911796e-01,-2.50997514e-01,-6.06571555e-01,1.66356891e-01,-3.37894559e-01,2.32313752e-01,5.12685418e-01,7.04004541e-02,-2.51307398e-01,6.41300576e-04,-1.46694386e+00,7.01302648e-01,-4.96819884e-01,-2.97829866e-01,-3.21868747e-01,-5.48544228e-01,2.06942141e-01,-3.66044223e-01,-2.50357658e-01,8.89264882e-01,1.00008033e-01,1.97767422e-01,-4.22170728e-01,-4.92165595e-01,-5.79827070e-01,3.28753471e-01,2.92940170e-01,-1.76026553e-01,-1.44002497e+00,2.02780724e-01,2.23967042e-02,-1.13810606e-01,-8.79819393e-01,-1.02361369e+00,6.58606589e-01,2.73046762e-01,2.23230615e-01,6.49935976e-02,3.14997256e-01,-1.34792352e+00,4.97855693e-01,-1.33866161e-01,8.52787673e-01,-3.20863038e-01,4.53846693e-01,-3.08676660e-01,-6.89715087e-01,2.14548737e-01,1.00057662e+00,-6.80762708e-01,2.48846009e-01,-2.65416056e-01,-9.30440366e-01,-6.36325836e-01,3.87015902e-02,-2.52499700e-01,-1.90721992e-02,3.47763419e-01,8.74829814e-02,-7.98588544e-02,-3.35877798e-02,2.39961848e-01,-1.25591950e-02,6.14243783e-02,-4.96838421e-01,-5.73918819e-01,-3.46861303e-01,2.15883926e-01,7.38309845e-02,-2.06728995e-01,3.18785787e-01,5.83439767e-01,-7.25774527e-01,5.65327585e-01,-5.63918389e-02,-2.08849117e-01,1.34481415e-01,-6.55173361e-01,-1.59793124e-01,-1.47297651e-01,-5.74318171e-01,6.28783286e-01,4.59490448e-01,4.55160618e-01,3.98904443e-01,5.55465579e-01,2.57416248e-01,-6.10560477e-01,-5.38755059e-01,9.35902521e-02,-4.95260268e-01,2.95966774e-01,1.90953210e-01,3.07933718e-01,1.60590351e-01,-7.99778223e-01],[6.84306026e-02,4.36475985e-02,5.94598413e-01,7.27032900e-01,-1.22203715e-01,5.13326377e-02,-2.04471022e-01,-6.13098621e-01,2.11414665e-01,1.37105495e-01,-8.07246491e-02,-1.36887282e-01,6.98204100e-01,-5.42870939e-01,4.04527187e-01,-2.61013329e-01,-6.80419326e-01,-4.70713288e-01,1.22780001e+00,1.07172120e+00,1.67660207e-01,-1.42192524e-02,1.67279944e-01,-4.26390953e-02,1.75659075e-01,3.04677278e-01,3.57901245e-01,3.94788742e-01,3.20094764e-01,-7.38328934e-01,-4.18482780e-01,-3.03866297e-01,4.40098822e-01,3.52901109e-02,-7.36009955e-01,-6.67973578e-01,3.93697619e-01,4.97423410e-01,-3.11046988e-01,1.11859846e+00,-3.23984146e-01,-1.76244736e-01,-5.04703045e-01,1.51191860e-01,-9.13440228e-01,-1.03750908e+00,5.38769841e-01,5.64232588e-01,2.83982068e-01,-5.49265034e-02,-3.54692787e-01,-2.70310372e-01,2.87472248e-01,-2.44083107e-02,-4.37168598e-01,-8.55026767e-02,2.73323596e-01,-1.75515190e-02,-2.66260952e-01,-8.28178287e-01,-2.22708017e-01,-3.85824114e-01,1.63009918e+00,1.27047324e+00,4.13697094e-01,2.53619105e-01,-4.57700118e-02,-3.82744819e-02,8.33102822e-01,6.23860717e-01,-1.13460720e+00,-4.16524410e-01,3.15323770e-01,6.72916472e-02,-4.60862428e-01,-5.91418028e-01,4.82628077e-01,-2.57443398e-01,-3.03765327e-01,4.32985574e-01,2.78128013e-02,1.48690566e-01,6.29898250e-01,1.35860252e+00,4.14565206e-01,-2.63499860e-02,-1.05574465e+00,-8.50428343e-01,-1.12947002e-01,-2.01351464e-01,1.84879661e-01,4.65688929e-02,-2.56216973e-01,-1.56494439e+00,4.77778055e-02,-7.58487433e-02,-1.06744252e-01,2.82628089e-01,-1.43271708e+00,-4.50513102e-02,-3.43557566e-01,3.13220024e-01,2.14377642e-01,4.35144454e-01,6.39408156e-02,1.76718265e-01,4.23964679e-01,-3.57268125e-01,3.53011250e-01,-1.36270106e-01,-3.44968662e-02,2.75877684e-01,3.00855070e-01,-3.77303332e-01,-1.26154765e-01,1.63869500e-01,2.69712329e-01,2.39266068e-01,-7.70182535e-02,-1.20534785e-01,1.81817766e-02,-2.34619826e-01,4.82875168e-01,-1.88847169e-01,1.32172510e-01,-3.10940325e-01,3.15765083e-01,5.15334666e-01,-3.54001850e-01,-1.09448381e-01,2.82403361e-02,1.20330751e-01,4.65475291e-01,-4.03334014e-02,-1.28555489e+00,-1.54052675e+00,3.15111913e-02,-8.21103603e-02,-1.22155577e-01,-4.43298876e-01,-1.60232987e-02,-3.18899333e-01,-7.18984082e-02,5.40982127e-01,4.57619399e-01,-1.52090833e-01,-1.04681873e+00,3.82229015e-02,-1.15967818e-01,4.76496756e-01,1.32450029e-01,2.88231283e-01,6.62524775e-02,4.34643567e-01,4.08958137e-01,5.20476401e-01,3.02994043e-01,1.99408978e-01,-9.29793194e-02,9.00191143e-02,-2.67077148e-01,-2.95066554e-02,-1.70999989e-01,4.39770639e-01,4.54663970e-02,2.98960835e-01,1.35549414e+00,3.37265402e-01,-2.04298645e-02,-4.50146198e-02,-5.04741669e-01,7.76246712e-02,4.11404043e-01,-3.34776454e-02,-4.87249345e-01,5.25066257e-01,-2.88910985e-01,2.38566801e-01,3.72411728e-01,6.91098273e-01,-7.81407058e-01,-1.01868719e-01,2.20892839e-02,-5.32531762e-04,8.98002803e-01,1.31631649e+00,-4.03465420e-01,-4.10856128e-01,5.56111000e-02,3.90822394e-03,-8.71722698e-02,1.65600762e-01,6.87166452e-01,2.47833371e-01,-6.97656274e-02,5.49002528e-01,-5.61907180e-02,-2.74986699e-02,1.75475806e-01,-3.69176954e-01,-1.73445597e-01,3.57247591e-02,-5.26988864e-01,-4.83288199e-01,-5.71822822e-02,7.44753659e-01,-5.18122017e-01,-6.85214877e-01,9.04129148e-02,-4.04990077e-01,5.12741804e-01,-6.97899610e-02,-2.57903904e-01,-3.56989652e-01,4.55824673e-01,9.63855609e-02,-1.01233371e-01,1.33041531e-01,5.53393602e-01,5.65080717e-02,-2.75492162e-01,-3.67738426e-01,-1.86913148e-01,2.12885275e-01,1.22448415e-01,-2.45591789e-01,-9.64860797e-01,-1.28176081e+00,-2.30506912e-01,-2.70839870e-01,2.20770482e-02,2.41672114e-01,-5.63456655e-01,-5.92733204e-01,1.27789810e-01,9.64711607e-02,3.25437710e-02,1.29836023e-01,1.91776492e-02,-3.74756120e-02],[-0.48065215,0.24968624,0.22006927,0.24474826,0.35193843,-0.12415422,0.4701009,-0.4297407,0.33855364,0.5767225,-0.5588249,0.3232099,0.16449967,0.20203945,0.0776508,0.503305,-0.01851968,-0.02818952,-0.06440601,0.2704215,0.33666804,0.06575261,0.7959806,-0.4574693,0.02948869,-0.432091,-0.0103349,0.1980012,0.19151329,-0.23673938,0.5079211,0.19683118,0.26598132,0.55663157,0.30504555,-0.17071646,-1.0635606,0.00522113,-0.60272115,0.95764804,-0.65178,-0.37387627,-0.96147114,-0.01026982,-0.37862438,-0.22764811,-0.7556177,-0.4645267,0.55114377,-0.1798871,-0.6451691,-0.8815048,-0.53202814,-0.14394163,0.25672162,0.2672464,0.494677,-0.4362001,-0.39998034,0.12973566,-0.48287037,0.5934562,-0.15453397,-0.91105294,0.43360803,-0.55193865,-0.07101622,0.51985115,-0.9519448,0.20709369,0.18252677,0.16001406,0.00347107,-0.04284045,-0.21046425,0.19674197,-0.25354204,0.37370872,0.19737329,-0.07231575,0.40555328,0.40538344,-0.61211246,-0.66579956,0.35461172,-0.2651026,0.2429261,0.11847956,-0.003869,-0.48688287,0.4825243,-0.34775883,-0.6057733,-0.3656634,0.33899775,-0.62029934,-1.0074216,0.3302511,-0.35066065,0.47875103,0.31518522,0.87136406,-0.61517215,-0.23878439,0.54944855,0.1437891,-0.60823154,-1.7517645,0.06648187,-0.10698006,0.32930088,0.07799376,0.31511867,0.7747757,-0.19378482,-0.10503554,0.4849104,0.2174204,-0.12734096,0.6818634,0.04461593,-0.27577072,0.5416969,0.11845377,-0.83469373,0.13193864,0.3340565,0.38383466,-0.42628643,-0.3461003,-0.63678175,0.16633976,-0.55043167,0.08811641,0.14297907,-0.34653184,0.2648196,0.38348255,0.3017174,0.20674859,-0.6049664,-0.03136335,0.39846075,-0.08885068,0.45737383,-0.24762225,-0.03009602,-0.25480145,-0.38060036,0.15422408,-1.1745371,0.53473186,0.1132127,-0.03770012,0.54355496,-0.6835363,0.95072854,0.20025381,-0.3651028,0.2836895,0.15093973,-0.30964383,-0.45611307,-0.0497662,-0.2887578,-0.3126367,-0.37108532,0.26524356,-0.27992648,1.0785744,-0.2196053,0.06439215,0.2564051,-0.02149275,-0.19374162,-0.18879543,-0.309022,0.03227523,-0.54858375,0.48864815,0.9469795,-0.42380154,0.265309,0.4487016,0.46656874,-0.09432588,0.30119166,0.8063782,0.32776454,0.7412573,-0.02300263,-0.7607871,0.519408,-0.47213084,-0.6052876,-0.11106779,-0.02287172,0.18698348,-0.20529655,0.08543769,0.53592813,-0.47593603,-0.5384754,0.67120194,-0.899105,-0.20365056,0.1163512,-0.1539148,-0.56549364,-0.3672893,0.1917602,-0.23314267,0.29375857,-0.76672876,-0.6797504,0.02202456,-0.53288907,0.4036513,-0.51972765,0.563505,-0.3789584,1.0779485,0.00916834,-0.33070946,-0.19037695,-0.3499036,-0.19942829,-0.55475247,0.36881715,-0.19656804,-0.2053764,0.34324148,-0.00202891,0.49724525,-0.02933908,0.19142096,0.08914009,0.01313497,0.51773405,0.65565413],[-7.18209818e-02,2.15014905e-01,4.72222269e-01,7.65332937e-01,-5.58206320e-01,2.05732986e-01,-3.39015156e-01,1.67599171e-01,1.78741604e-01,-3.47565830e-01,7.21150458e-01,-2.59198397e-01,8.54870260e-01,-5.87507546e-01,-1.08773494e+00,5.43926060e-01,1.14793599e-01,-5.16091585e-01,-2.76946928e-04,-2.49489918e-01,-1.00812577e-01,2.29058549e-01,-7.76528493e-02,-1.16008617e-01,8.87139142e-02,5.80504775e-01,-1.43883884e+00,-7.69127667e-01,1.99884195e-02,6.13364018e-02,3.76168549e-01,4.19523299e-01,-1.03892691e-01,7.44167924e-01,8.02911341e-01,6.65122569e-02,1.38337716e-01,-3.01411241e-01,-5.73161878e-02,3.13836522e-02,4.68766570e-01,2.02564821e-01,-6.87524498e-01,1.62287205e-01,-2.80434072e-01,4.22629833e-01,6.48570713e-04,-6.74414337e-02,2.79132407e-02,2.21501306e-01,-2.83172488e-01,6.65959716e-01,-4.33901429e-01,7.06819832e-01,-4.45355356e-01,-2.16530934e-01,5.33603609e-01,-3.43709767e-01,-1.62234202e-01,-1.57603204e-01,-1.22486956e-01,6.73465788e-01,2.96342000e-02,-2.20273986e-01,-1.86722636e-01,4.72095549e-01,5.40961586e-02,-1.44746348e-01,3.63598645e-01,5.57470441e-01,-1.68197557e-01,-3.06073308e-01,1.69862155e-02,-1.37680605e-01,1.39531940e-01,-1.19019717e-01,-7.69384801e-01,5.16963238e-03,-4.68703173e-02,-2.57937402e-01,3.05303842e-01,5.13542950e-01,5.51833212e-01,6.80027008e-02,-5.53303778e-01,2.04211116e-01,4.64981526e-01,3.68610881e-02,-2.67892599e-01,8.16415101e-02,-5.67686081e-01,-2.70376205e-02,-1.94759265e-01,4.28925335e-01,3.14070255e-01,-4.89784852e-02,-4.88767028e-01,3.59740227e-01,5.16119242e-01,-4.67811078e-01,-1.22054172e+00,6.28682077e-01,3.01997632e-01,-4.02840585e-01,7.45696068e-01,-2.00037301e-01,1.16664849e-01,-1.00235514e-01,-7.06211865e-01,2.28050545e-01,-6.07868433e-01,-8.23880374e-01,5.87907255e-01,-3.25109988e-01,2.11905405e-01,4.39042032e-01,1.93896722e-02,-8.04794207e-02,-7.31006086e-01,-8.64486814e-01,-1.22391596e-01,4.36933130e-01,-1.63051784e-01,6.40462577e-01,-1.67010754e-01,-3.66607964e-01,8.50791931e-02,-7.91653574e-01,-5.98065019e-01,-5.22894144e-01,-4.49456662e-01,1.05718279e+00,7.57650256e-01,-3.41021538e-01,2.18271315e-01,-1.79897547e-01,3.87094736e-01,-1.06225479e+00,5.39862752e-01,4.95026499e-01,-1.35920393e+00,8.94142315e-02,-4.51710582e-01,-3.92102808e-01,2.61423051e-01,-5.20138204e-01,-8.71840417e-01,-6.02530122e-01,-4.40115243e-01,3.34139079e-01,9.60381806e-01,4.50951001e-03,6.88626230e-01,-1.73495263e-01,2.88888693e-01,2.88902432e-01,4.18176055e-01,-8.73412549e-01,-7.40585625e-02,4.08679187e-01,7.18003273e-01,-4.02264267e-01,-3.82465839e-01,-1.94835052e-01,1.22652873e-01,1.87457129e-01,-5.73022425e-01,-1.35322452e-01,1.59208477e-02,-9.20814335e-01,-1.09230268e+00,-2.09226251e-01,6.45458221e-01,-6.08136691e-02,8.60346630e-02,-2.68024243e-02,3.19837332e-01,-3.90580058e-01,2.22572759e-01,-1.40653387e-01,-6.37325823e-01,-4.69199926e-01,-1.03172436e-01,3.24096948e-01,-2.48027563e-01,1.46880090e-01,-9.82972085e-02,3.16156708e-02,-1.86114222e-01,5.27397752e-01,-7.22820878e-01,-3.63826334e-01,-4.81901288e-01,6.88589454e-01,1.61502272e-01,3.94167304e-01,2.85950661e-01,-6.02109469e-02,-1.02295533e-01,-1.91018134e-01,-1.41446030e+00,6.28886893e-02,-9.84830022e-01,9.07396972e-02,3.38282734e-01,4.50630993e-01,-7.68971860e-01,6.52452171e-01,-3.55676502e-01,1.40673637e-01,-3.00376505e-01,2.80491263e-01,-3.54323328e-01,-3.73531133e-01,5.70143700e-01,-9.66147110e-02,3.55087221e-01,-2.38250732e-01,-6.29017472e-01,-9.61194634e-02,2.45291144e-01,-3.74570005e-02,-6.33745670e-01,-1.02689195e+00,2.99631357e-01,-2.13935897e-01,-1.08635537e-01,2.41085529e-01,2.46044382e-01,1.51186794e-01,-1.70298040e-01,-2.13825926e-01,1.66285783e-01,1.22362196e-01,-4.96115237e-01,3.04736614e-01,-5.68477018e-03,6.89208806e-02,1.83108881e-01,-4.09557760e-01],[-0.03987588,-0.4147896,0.14382054,-0.01820773,-0.49330705,0.12635145,-0.17645484,0.17403421,-0.6075724,-0.15820865,-0.36323997,-0.624846,-1.0261424,0.16012937,0.08918205,0.49919772,0.12342696,0.2881017,-0.12478931,0.08404318,0.17728226,-0.35549203,0.02828404,-0.7334142,-1.6358898,-0.5595174,0.6906593,-0.39878654,0.36617735,0.7912836,0.34260634,0.32500386,-0.18532425,0.40323207,-0.33404544,0.05303915,-0.7297033,-0.24164502,-0.04437243,-0.75510806,0.00769702,-1.1017008,-0.3338401,-1.175513,-0.00203935,0.7339413,0.5006825,0.40920374,-0.08245233,0.02033569,-0.26747236,0.14679852,0.0416569,-0.19765341,0.3747335,0.10470678,0.25988233,-0.21258475,0.08874812,-0.7801966,0.40470767,-0.16691235,0.5793014,-0.4450204,-0.31036583,0.00278919,-0.0489873,-0.66662246,-0.96027535,-0.6138538,0.06134564,0.1446357,-0.28153798,0.07462473,-0.40042463,-0.17972441,-0.3321734,0.54664546,-0.4157724,0.47236037,0.20414774,0.08881065,-0.2330726,-0.03217896,-0.82791525,0.5224092,-0.17459045,-0.07218037,0.5154407,-0.5700414,-0.25902358,0.07550187,0.21881191,-0.20220329,0.5555629,0.04498972,-0.25802928,0.22711292,-0.2717253,-0.06052062,0.12766863,0.8195127,0.74144435,0.08560158,-0.6978463,0.9689841,0.16159944,0.29098168,0.33801562,0.67786217,0.64657074,-0.6238246,-0.14805733,-0.27860105,0.8439522,-0.2820472,-0.19432744,-0.29183647,-0.5698845,0.260469,0.16382018,0.39391923,0.7006923,0.33918023,0.4067039,0.0233636,0.62947416,-0.29612008,-0.12331439,0.63659644,-0.09259592,0.21055594,0.49585193,-0.42938876,0.38483799,0.18242505,0.22522905,-0.2763756,0.22304629,0.70291287,-0.52395624,-0.11725281,0.34196523,-0.51124835,0.32776734,-0.48096105,0.23182993,-0.37237394,-0.12848559,0.0702315,-0.9667186,0.2090196,0.53960085,0.88372105,0.10205182,0.47401807,-0.45718083,0.97668666,0.41208634,0.03809825,0.7130143,0.51176643,0.05740776,0.18700135,-0.2866086,1.0410992,-0.2744524,0.24781184,0.22635485,0.5946706,0.11295926,0.38849366,-0.54732615,0.5584358,-1.5746784,-0.595522,-0.23240674,-0.18378544,-0.756837,-0.40740532,-0.34364313,-0.3576591,-0.34647,-0.6336608,-0.45071313,-0.032255,-0.08700759,0.3892523,-0.514185,-0.21520345,0.0369133,0.02945618,-0.927261,-0.04467446,-0.5340459,-0.0099778,0.17711943,-0.05302214,0.3968763,0.40233636,0.46028644,-0.45610243,-0.13234334,-0.47621718,-0.3133095,0.65294725,-0.8342665,-0.11835658,0.04029395,-0.42062896,-0.3579875,-0.49059117,-0.21475132,-0.04674323,-1.1397189,-0.4426095,-0.7920168,0.5657096,-0.30247056,0.25032455,0.5035112,0.24916446,-0.431923,0.3916775,0.8659714,0.1138109,-0.4222579,-0.3056309,-0.85204357,-0.00928304,-0.19232617,0.27127668,0.37579778,0.15784433,0.13599092,-0.70492977,-0.08975398,-0.09111574,-0.4088787,0.22621727],[3.8776380e-01,-4.4917846e-01,6.5248263e-01,5.0969315e-01,-1.7997761e-01,-6.2616184e-02,3.5538089e-01,-6.7535824e-01,-5.7364833e-01,2.1259671e-01,3.7345466e-01,-2.3792168e-02,3.9180520e-01,-4.7377038e-01,-4.1608417e-01,-1.0679600e+00,-1.6348307e-01,2.6965845e-01,-6.3398373e-01,8.2107060e-02,8.4854051e-02,-4.0706632e-01,-4.9986979e-01,5.3764474e-01,3.5528153e-01,-3.4969246e-01,-6.1366183e-01,-1.0760986e+00,-4.3474287e-01,-4.6956232e-01,-2.7246886e-01,5.0152707e-01,2.6818717e-01,5.5447066e-01,2.0252536e-01,-4.1300446e-01,4.1973081e-01,3.6575314e-01,4.7737053e-01,-1.3741870e-01,-2.9153418e-02,-3.3781141e-01,1.9501740e-01,-3.9779681e-01,-3.1618407e-01,-5.3532964e-01,1.8467772e-01,1.1339277e-01,7.8377587e-01,2.6981133e-01,-3.6225867e-01,2.3139982e-01,-4.6321395e-01,-2.9104033e-01,3.2232109e-01,1.5784672e-01,-8.0488622e-01,8.6560047e-01,-1.2030061e-01,2.7555013e-01,-9.9324214e-04,9.5309820e-03,6.9776624e-01,-2.7589977e-01,4.7778755e-02,6.3140321e-01,4.8328150e-02,2.3918252e-01,-1.7485449e-01,1.9562760e-01,-2.8450656e-01,4.2050028e-01,3.4368867e-01,-4.8644269e-01,-4.4934347e-01,-7.7483736e-02,2.3903728e-01,-3.4567973e-01,-4.0023968e-01,1.4208992e-01,7.8379917e-01,-2.5873443e-01,-1.0525406e-01,3.3440599e-01,-1.9707927e-01,-6.8395302e-02,-1.9283029e-01,-4.9500480e-01,-2.4054974e-01,-3.1999052e-01,5.4473257e-01,-1.0927233e+00,-3.9597306e-02,4.0508285e-02,-9.2782640e-01,1.8155302e-01,4.3941733e-01,8.8733777e-02,1.1902653e-01,1.6893935e-01,-2.8476419e-02,-1.0633956e-01,6.0173750e-01,2.1898271e-01,-4.5223355e-02,3.3784583e-01,4.6808046e-01,-3.5195363e-01,-3.2131198e-01,-1.5216626e-01,-2.6389152e-01,-5.8141053e-01,-4.3178353e-01,4.7090796e-01,-6.8404265e-02,6.0200328e-01,2.9963540e-02,3.8497284e-01,4.5607854e-02,5.9302795e-01,1.3348347e-01,1.8022998e-01,3.8756779e-01,4.1394797e-01,4.2943597e-01,3.0985478e-01,-1.5918732e-01,-3.2612953e-01,4.8164418e-01,-6.4851850e-02,-1.6282152e+00,7.3670423e-01,-1.8474658e-01,-5.8297879e-01,-6.6126429e-02,3.2394847e-01,-2.2799027e-01,-1.7305210e-01,1.9455522e-01,-9.3319738e-01,6.5597254e-01,-4.1887052e-02,9.4624972e-01,-1.0804220e-01,-2.8679478e-01,6.2437063e-01,-1.2412248e-01,-7.2741762e-02,-3.0269915e-01,-4.2568114e-01,-1.4566211e-01,4.7675398e-01,-1.2623905e-01,-4.6716869e-01,1.9924752e-01,-3.3948842e-01,-3.1209540e-02,-2.1695836e-01,7.5042576e-02,4.2163509e-01,5.8578634e-01,-3.7131530e-01,-2.3423624e-01,-1.1843328e-02,3.6593072e-02,7.3048973e-01,5.2724820e-01,-3.4359872e-01,-1.1005191e-01,-4.0605754e-02,-1.6303860e-01,-3.0154973e-01,3.5357770e-01,-6.8961555e-01,1.1902130e-01,4.4805762e-01,-5.8282632e-02,1.1751322e-01,-1.0028871e+00,-1.3776632e-01,-4.7336567e-02,1.3509177e-01,3.0454764e-01,-5.1566023e-01,-4.5855460e-01,4.6958271e-01,8.9345419e-01,5.0674486e-01,-1.3237472e-01,2.7887979e-01,3.5648206e-01,-4.5589796e-01,-5.4611278e-01,9.5518947e-01,-1.9967104e-01,-1.3802156e-01,-9.5917486e-02,1.8160267e-01,-5.3707581e-02,-3.3608952e-01,3.9906448e-01,-4.7140154e-01,4.1349158e-01,-3.9922231e-01,9.4740468e-01,-7.9889005e-01,4.4530025e-01,-1.4810672e-01,-9.6496522e-02,8.2196377e-02,1.0396729e+00,-5.6970954e-01,-5.0926459e-01,-2.5087250e-03,-6.6902012e-01,-7.5757056e-01,5.3364798e-02,-1.2950010e-02,-8.1739283e-01,-2.0247868e-01,6.3460273e-01,-8.7242648e-02,-1.0301164e-01,-3.3786276e-01,-1.7714772e-01,3.8707774e-02,-4.5559434e-03,-7.2280186e-01,8.6695492e-02,-6.3838810e-01,7.7116758e-02,-3.4561363e-01,2.8198406e-01,5.8745599e-01,4.1537169e-01,-2.6863268e-01,-2.1644851e-02,1.9963202e-01,-6.2645793e-01,1.1373347e+00],[-0.5157844,0.37280804,-0.49063292,-0.07242695,0.33947808,-0.02076269,0.43675607,-0.7126852,0.20397224,-0.36295068,-0.0015743,0.27033687,-0.13156337,-0.0072683,-0.02720986,-0.3737521,-0.257601,-0.5113559,-0.24290957,0.16020606,0.13887194,-0.5932582,0.43880653,-0.05411765,0.19138962,-0.22894734,0.15473236,-0.6525729,-0.6481006,0.2879411,-0.08231919,-0.23242843,-0.8970164,0.06424984,0.34269524,-0.76585674,0.21291214,-0.14138785,0.12761606,-0.07224903,-0.31449077,0.403704,-0.20627227,-0.4423004,0.25477254,-0.726098,0.06628858,-0.18978828,-0.3999737,0.17415145,-0.1513782,-0.18757425,0.03471329,0.06724079,-0.2540865,-1.2424184,-0.00473848,0.11197063,-0.3427953,-0.13434064,-0.93076754,0.23844118,0.1233677,0.61049026,0.24939021,-0.07178766,0.70671767,0.06026268,-0.59401834,-0.04078778,-0.33957595,0.59637535,-0.3368963,0.10990969,0.53747207,0.19687034,-0.11213298,-0.5045599,-0.40407526,0.17772697,-0.97270995,0.01981849,0.07100782,-0.10321458,-0.29040432,-0.52409977,0.02427198,0.4273814,0.23911813,0.24262102,-0.5205926,-0.7573729,-0.04310505,0.38683608,0.14735265,0.4765036,-0.7287839,-0.8651841,-0.44315547,-0.12897918,0.0289007,-0.28942516,0.1766949,0.09372234,0.02995739,0.8065899,-0.15860885,0.95499384,-0.71127534,-0.9680221,-0.23826936,-0.26906532,-0.65104425,-0.16600437,-0.40067008,0.36260355,-0.07622344,-0.48686117,0.36586812,0.06117082,0.03689631,0.12103361,-0.22211115,-0.05592306,-0.97244173,-0.78884757,-0.08014239,0.11979024,-0.38167438,0.17785193,0.19557913,0.5994777,-0.6694355,0.32130745,0.05296354,-0.17301308,0.42235678,0.44459867,-0.6627972,0.37405756,-0.65706104,-0.2397701,-0.42389408,-1.0026714,-0.29830107,-0.70184124,-0.44860438,-0.13875432,0.09491257,-0.30245644,0.04231547,0.2417548,0.0820048,0.69425833,0.9295551,0.79871005,0.5488625,0.36900482,-0.1748537,0.6398905,-1.0994747,-0.47369722,-0.36242503,0.61482453,0.0457409,-0.35958216,0.19120766,-0.01706158,0.45753592,-0.46046495,-0.33758888,-0.04659567,0.32290086,-0.39131182,0.97539604,0.33354536,0.40972507,-0.36146215,-0.62834954,0.22089341,0.3351863,0.40451592,0.3314022,0.30312073,0.46274245,-0.11792473,-0.49142078,-0.17011791,-0.41248125,0.00336882,-0.2601485,0.61604846,-0.11412171,-0.50283796,0.33591485,0.16079189,-0.64566916,0.1798632,0.22875285,0.21427616,-0.43506894,-0.65673643,-0.34631643,-0.09674221,-1.145794,0.12093388,0.36757153,-0.09506845,0.46409607,0.38928217,0.20327649,0.09819587,0.9279523,-0.04306235,0.09117963,0.15590598,0.502471,0.39869913,-0.42867398,-0.22617167,0.32114348,-0.0264189,0.40012562,0.30973318,0.53718543,-0.5909753,0.28498754,0.94059956,0.19675142,-0.322796,-0.63586175,-0.01395959,0.08784325,0.10616368,-0.2976086,0.9010611,0.73922694,-0.02551035,0.4099536,-0.04528021],[-6.32344723e-01,2.69896090e-01,-2.48953491e-01,1.92292675e-01,-3.61645073e-02,-7.67864704e-01,-3.41852278e-01,-4.42072123e-01,-3.30249548e-01,-3.23255092e-01,-4.62595761e-01,-1.58186018e-01,2.19865844e-01,3.31329674e-01,6.35109007e-01,-1.32499546e-01,2.71601319e-01,-4.83494461e-01,-5.40414333e-01,8.11597928e-02,-5.01647949e-01,9.01951417e-02,-6.88409746e-01,6.52182758e-01,5.24180174e-01,7.58216754e-02,9.21460018e-02,2.43337631e-01,-2.42974056e-06,-2.17814177e-01,-2.16437683e-01,-9.79793429e-01,3.38571846e-01,2.87592083e-01,1.52438715e-01,2.16315761e-01,-4.33133781e-01,-2.12481529e-01,-5.75408936e-01,-7.08784103e-01,2.46267349e-01,2.62091815e-01,7.13028908e-01,-5.74541867e-01,2.66253710e-01,-1.21671192e-01,4.63051975e-01,-7.69386888e-02,-9.83329415e-02,-1.24496683e-01,-3.17746550e-01,5.83571911e-01,4.10353392e-01,-4.94407266e-01,5.63727200e-01,1.66262850e-01,2.30259955e-01,4.85558897e-01,-3.67633313e-01,-1.98368847e-01,-1.09708637e-01,4.92959917e-01,-2.52714097e-01,1.41182974e-01,2.51196176e-01,-3.58158350e-01,1.71217710e-01,-3.06950182e-01,-2.41486244e-02,9.19319317e-02,-3.98657590e-01,1.88259512e-01,2.06439033e-01,-7.03395665e-01,4.59245175e-01,-3.91347080e-01,-3.51528436e-01,1.47812888e-01,4.03190881e-01,2.75731236e-01,2.32198089e-01,5.33951223e-01,6.87052071e-01,2.29060739e-01,2.84169078e-01,4.43290472e-01,-6.67774975e-01,-2.99422830e-01,-7.82791898e-02,-3.30928206e-01,4.72675413e-01,-3.36457193e-01,-5.79642579e-02,4.76313680e-01,6.49693131e-01,2.14486659e-01,2.09435493e-01,1.52944326e-01,2.63625830e-01,-4.45194423e-01,1.28168732e-01,-2.42579177e-01,-1.35477543e-01,4.71204698e-01,1.16951205e-01,1.11154519e-01,6.55663788e-01,4.35462087e-01,-4.51110482e-01,6.25162274e-02,8.81933495e-02,-6.94553494e-01,-1.12899101e+00,9.39246044e-02,-3.03023178e-02,-4.84602481e-01,-4.61918831e-01,6.77763596e-02,2.45542109e-01,-8.98715705e-02,1.25278816e-01,-5.02589107e-01,1.61848679e-01,3.82338881e-01,1.59352366e-02,3.67784593e-03,-2.48284996e-01,3.31226885e-01,4.77566898e-01,-1.01250446e+00,-7.83384800e-01,4.09645051e-01,-3.96728098e-01,1.59428865e-01,-4.54956666e-02,1.16098657e-01,1.86550140e-01,-2.62890488e-01,-8.59284252e-02,-5.00804305e-01,-2.13843416e-02,1.77802704e-02,-1.71051115e-01,-1.96410939e-02,2.26507202e-01,-2.34652802e-01,-9.12615180e-01,2.28674542e-02,3.49598229e-01,-2.09543958e-01,-5.15445471e-02,1.77738011e-01,-5.93647361e-01,1.73025578e-01,-1.47027627e-01,8.06117132e-02,5.31929135e-01,-4.12993282e-02,-5.06737947e-01,2.02664137e-01,-1.09261823e+00,3.81609082e-01,5.44776022e-03,1.73589196e-02,-8.89828354e-02,1.00159657e+00,4.25677508e-01,-9.44382474e-02,-1.37807190e-01,4.14675593e-01,2.68893898e-01,-4.09155756e-01,-2.31282473e-01,-4.37883019e-01,-6.37548983e-01,8.90434086e-01,1.72513261e-01,-4.49750841e-01,-9.89825726e-02,-4.13620889e-01,-1.44591975e+00,-1.00318752e-01,-5.86994827e-01,1.39794752e-01,7.22995162e-01,-6.33760214e-01,-2.28328735e-01,7.71985427e-02,-5.88282108e-01,1.51198968e-01,3.06888521e-01,-3.42920095e-01,-5.63536108e-01,-1.22301362e-01,1.47701427e-01,6.11650467e-01,4.33506191e-01,-3.24365556e-01,6.98196769e-01,-4.23784778e-02,-5.40793180e-01,7.43100047e-01,-3.37078981e-02,-8.63317624e-02,-3.18015702e-02,-3.22191119e-01,5.42034030e-01,-5.70091784e-01,1.70367733e-01,5.17159700e-01,-6.27024114e-01,-2.36311674e-01,4.07235116e-01,-3.11433941e-01,-7.50067711e-01,1.39790297e-01,-7.00073302e-01,-7.55821839e-02,2.68487632e-01,1.27996400e-01,-2.54483223e-01,2.98014227e-02,-1.29212076e-02,-4.88587767e-01,-3.51632297e-01,-5.67264795e-01,-3.73086065e-01,2.32828692e-01,5.35348415e-01,1.60381094e-01,-5.67672849e-01,7.43911088e-01,-7.81067312e-01,-2.09723115e-01,4.65842783e-01,-1.74818501e-01,-8.26904893e-01,-7.48804677e-03,4.31432933e-01,-7.56809115e-01],[-0.6047174,-0.29261148,0.7281357,-0.29825962,0.38529268,0.14733453,0.3096251,-0.12238533,-0.45077455,-0.23615399,-0.5972807,0.200993,-0.20787525,0.09276373,0.19370459,0.183346,0.08953357,0.6546468,0.5752781,0.10118706,-0.780359,0.36089915,0.03885717,-0.25488296,-0.0589808,-0.80870754,0.6156124,-0.36766103,0.0391084,0.51252794,0.6072729,0.26036286,-0.01036124,0.35059726,-0.46919197,0.37080944,-0.10772046,0.07364222,-0.13686013,0.1065338,-0.23800647,0.16733986,0.21186633,0.501066,-0.13722208,0.05704902,-0.24832119,0.09022886,0.5974432,0.0116954,-0.5895037,-1.2369761,0.13091764,-0.10637817,0.91303426,-0.4016129,0.09907249,0.92934865,0.18379875,-0.08954246,0.68995905,-0.00198635,0.05765443,0.39877293,0.44620374,-0.7043882,-0.7341412,-0.21067847,0.3158807,0.08731993,0.5388226,-0.1427654,0.19992696,0.0641963,-0.5598688,-0.6358696,-1.2701321,0.0607131,0.02386685,-0.77681905,-0.03002369,0.4814988,-0.15319568,0.2859296,0.7954654,-0.03110757,0.2777698,-0.1907047,-0.31304315,0.29747215,-0.0652281,-0.15618783,0.10757752,0.07113362,0.32952243,0.11286201,0.06843784,-0.59672487,0.26399887,-0.3543903,0.39821735,0.09383028,0.26723918,0.43690434,0.6384016,-0.228896,-0.36402628,0.19074868,-0.7637646,-0.8926747,-0.43774405,0.105758,0.39803407,0.02451369,-0.02054641,0.4122363,0.07871646,0.02009201,0.41097406,-0.43883812,-0.1116539,-0.0604591,-0.13343346,0.6504412,0.55185467,-0.8786876,-0.06753221,-0.27021816,-0.22432004,-0.36496454,-0.14017119,-0.4431727,0.08203436,-1.5598911,-0.44844022,-0.05638788,0.6264677,-0.1294379,-0.10221341,-0.3040118,-0.1144664,0.22487716,0.2004503,0.3228194,-0.623665,0.45017347,0.09059516,-0.33170715,-1.0873239,0.49189746,-0.14427355,-0.15062883,-0.04697,0.18439037,0.10978898,-1.0569851,-0.07613827,0.11124059,-0.60037583,0.90653676,-0.15459126,0.4907622,-0.67959034,-0.33245042,0.18489547,-1.1368829,-1.4883134,0.27182245,-1.0059355,0.52784747,0.02201805,-0.12451448,-0.6909686,0.01300133,-0.684362,0.5803264,-0.28934667,0.87614673,-0.4884193,0.588336,-0.04603041,0.90668076,0.41630384,-0.37864357,-0.3785709,-0.9848056,-0.6411121,-0.16141298,0.2848604,0.20986496,-0.28774673,0.31550637,0.43175802,-0.25341704,0.00727355,0.17021151,-0.06712501,0.23925723,-0.31207326,-0.06175579,0.56846356,-0.12695627,-0.01921002,-0.03861102,0.6377595,-0.07772203,0.18016778,0.41737133,-0.11059979,0.15286922,-0.38715714,0.26898375,-0.4531434,0.2036792,-0.03955673,-0.13610268,-0.06265827,0.13952833,-1.1521883,0.21381551,0.12988846,-0.27239484,0.49562606,-0.12348547,0.02228291,-0.5325995,0.30905962,0.06198854,0.30876327,-0.1311051,-0.65725833,0.4753272,-0.07739741,0.12003641,-0.22232984,-0.11277401,0.5179407,-0.7588972,-0.1950687,0.02132685],[-0.12308205,-0.35995218,0.3467463,-0.43396357,0.12727864,-0.1290591,-0.5888135,-0.5653332,-0.28744105,0.14300787,-0.0933442,-0.27196428,0.13452743,-0.07153994,1.075622,-0.19978139,-0.08552751,-0.24530858,0.29811817,0.07555119,0.19958605,-0.06504507,-0.398106,-0.774778,0.7442972,0.37449306,0.22915316,-0.49297935,-0.08843455,0.1909816,0.1098437,-0.39184466,-0.1799049,0.1719356,0.07395381,0.3725891,1.0742224,-0.29292673,0.17943168,0.01720509,-0.20441249,0.51888907,0.10030805,0.10815481,0.3751727,-0.0938236,-0.31285015,0.4099207,0.15221845,-0.7145042,0.18807973,0.03709837,0.49232933,0.22008069,-0.08358251,-0.44642764,0.34188145,0.09315708,0.841832,0.24601845,-0.3313688,0.2684798,-0.21264336,-0.25393707,-0.6295544,0.2761333,0.5238233,-0.3754264,-0.3759529,-0.9629706,0.748437,-0.23686081,-0.20478114,0.45358217,-0.5073767,-0.0793762,0.11968993,-0.12273043,0.44901207,-0.5114872,0.45825398,-0.30827647,-0.04713356,0.4633102,-0.12433453,-0.03713801,-0.6087437,0.35414195,0.4264172,-0.50840354,-1.3224624,-0.58636117,0.27344844,0.17308347,-0.52081794,-0.53058565,-0.13476768,-0.12885618,-0.2399246,-0.30404392,-0.39964077,1.103525,-0.35299465,0.0883456,-0.50250566,1.0432867,-0.67142737,0.00519038,0.02114744,0.09376109,0.05687306,-0.44022363,0.4696982,-0.4369983,-0.47007766,-0.43295807,0.21395676,-0.44769534,-0.05316797,0.03051931,-0.14270265,-0.52157664,-0.07520485,-0.1042149,-0.16536081,0.35450226,0.34639367,-0.08499747,0.19555691,0.31372854,-0.26312035,-0.26077607,0.46668273,0.4057975,-0.3353637,-0.6343186,0.13746168,-0.931358,0.21155334,0.06621489,-0.38483503,-0.08345848,0.21109962,-0.5939972,-0.16643573,0.605336,0.51443005,-0.20055218,-0.00692304,-0.58124423,0.6650445,0.26319373,0.786871,0.4091031,0.3814059,-0.32806045,-0.6749355,-0.07295418,-0.61039805,-0.3521675,-1.0072908,-0.17314063,0.42799878,0.07770339,-0.3172335,-0.07876921,0.3481772,-0.25044173,0.10569495,0.21982343,0.31485644,0.31546003,0.47058454,-0.18381648,-0.01053711,-1.2355283,-0.26434413,-0.4332496,-0.3052223,0.3885107,-1.0309055,0.27226058,0.1780055,0.3177802,-0.5572885,0.00227322,0.11880592,0.2343406,0.3090693,-0.6990839,-0.41290146,0.2454023,-0.12416128,0.2308237,1.1344423,0.56711966,0.7526881,0.76139075,0.19621192,-0.58532435,-0.42316306,0.13145898,-1.0318712,-0.06296758,0.43736526,0.48554087,-0.21421553,0.31953862,0.35564965,-0.13588779,0.23103842,0.03829061,-0.7478596,-0.21486883,-0.47103563,0.06808698,-0.5177177,-0.14221895,-0.08290917,-1.0189174,-0.6738417,-0.5619817,-0.06878795,0.42727336,0.33037066,0.34742886,0.40431648,-1.4641935,1.2411215,-0.22382067,-0.33232915,0.8580578,0.26421723,0.37065703,-0.212558,-0.5326937,-0.00644189,-0.9430279,0.30927998,-0.08321497],[-1.09129965e+00,-1.48328531e+00,-2.96731889e-02,6.38114989e-01,1.96955144e-01,4.66686785e-01,-8.31825376e-01,2.01236635e-01,-6.62176833e-02,4.89345975e-02,1.03880063e-01,-6.22038662e-01,-2.99296558e-01,7.33334005e-01,5.40949702e-01,4.67439778e-02,-3.61320108e-01,2.10971773e-01,-8.38074461e-02,2.09993213e-01,-5.31149030e-01,2.73079891e-02,7.82382637e-02,1.41801253e-01,6.98443830e-01,-4.81884509e-01,2.75130510e-01,2.94838220e-01,-1.24907863e+00,6.07235610e-01,6.20651841e-01,3.69573325e-01,-1.46796688e-01,9.13162008e-02,-5.96105278e-01,1.67813152e-01,6.32493198e-01,-4.91449863e-01,-6.12396747e-03,1.13276727e-01,-5.56081355e-01,-2.48370618e-01,-6.98610961e-01,-2.61029080e-02,4.47774619e-01,-1.15891419e-01,-3.04954559e-01,2.43032843e-01,-3.89144421e-01,3.81120354e-01,5.23460746e-01,-1.46559887e-02,2.85621554e-01,3.11541229e-01,3.98074627e-01,-1.96156502e-01,-1.33425534e-01,-6.41230226e-01,-5.51851332e-01,2.07491085e-01,-1.60562381e-01,-9.84611928e-01,6.31175995e-01,-3.28305751e-01,3.32961529e-01,3.61484319e-01,1.29867047e-01,-4.38447744e-02,-4.32403535e-01,-8.94788980e-01,6.54926956e-01,3.85022730e-01,2.29634449e-01,2.04747275e-01,1.13768913e-01,-3.85290682e-01,2.99085975e-01,3.60357855e-03,-9.60569322e-01,4.92556281e-02,2.94646531e-01,3.04352075e-01,-4.62098837e-01,-5.01480043e-01,-9.58984613e-01,-1.44106925e-01,-1.47068679e-01,-3.85788947e-01,2.12051272e-01,-2.42049694e-01,-2.95152128e-01,2.65287347e-02,-1.69608042e-01,-2.87445635e-01,-6.16432786e-01,-2.15753578e-02,-5.97396269e-02,4.97974724e-01,-4.63624299e-02,4.13315713e-01,3.51608127e-01,-1.34150073e-01,5.36701120e-02,-1.31221879e+00,-2.13066369e-01,2.24992335e-01,-7.83319652e-01,-2.13068798e-01,-3.06028932e-01,1.92171112e-01,-3.02588582e-01,7.71399677e-01,3.47820550e-01,-2.21217334e-01,-4.28983539e-01,1.83125257e-01,-1.05342254e-01,2.33449653e-01,4.90468591e-01,-2.08440408e-01,6.00426495e-01,-7.18437582e-02,-1.25398666e-01,1.08227089e-01,1.10269725e+00,2.36263201e-01,-1.79241747e-01,-9.47419703e-01,-4.84798104e-02,5.68187177e-01,9.84354764e-02,-2.53180385e-01,-6.60798848e-01,3.67601365e-01,5.65673746e-02,8.55260864e-02,-4.00241077e-01,8.02843794e-02,-4.48774725e-01,4.65703681e-02,5.21622002e-01,-2.90068984e-01,-3.56952578e-01,4.70334709e-01,-2.28086948e-01,-1.44938469e+00,4.90825959e-02,1.87604532e-01,-3.62755619e-02,5.77159464e-01,-7.27408826e-02,3.22764248e-01,-5.19427299e-01,-2.87031960e-02,4.76774350e-02,4.77870584e-01,-1.11707814e-01,-5.44665039e-01,-3.57324965e-02,-3.35363805e-01,-1.76429927e-01,3.03325057e-01,-2.12398648e-01,1.59526721e-01,8.18059519e-02,6.57346785e-01,-2.46784464e-01,-7.29772747e-01,-3.10771048e-01,1.16759814e-01,8.40973675e-01,7.99155533e-02,-5.62274933e-01,-1.42147839e-01,1.67131603e-01,-2.28101790e-01,5.57327345e-02,2.22964678e-02,1.23215306e+00,1.51654154e-01,-6.73159137e-02,-6.74598515e-02,4.36109863e-03,4.29110855e-01,-2.08074301e-01,-1.00508615e-01,1.24628372e-01,9.98648852e-02,-5.01555920e-01,-4.48990792e-01,7.52850652e-01,-5.43635190e-01,-3.54498774e-01,8.20912123e-01,-8.50900412e-01,-6.24680817e-01,-4.28224832e-01,2.06032813e-01,-3.64533812e-02,1.29065186e-01,-5.59105612e-02,3.31635386e-01,-2.51910716e-01,-3.87627661e-01,7.15349793e-01,3.81442428e-01,6.55661151e-02,-1.03231177e-01,8.53467464e-01,7.37372577e-01,3.92181486e-01,-2.19640017e-01,9.55117643e-02,-2.94788718e-01,-2.77016103e-01,-2.49580860e-01,4.26432602e-02,-5.33800840e-01,7.02133656e-01,-6.59212530e-01,-1.48654044e-01,-1.35314523e-03,6.87148988e-01,-2.53355771e-01,-5.02144575e-01,-1.23201244e-01,4.41724151e-01,2.30647415e-01,4.97392528e-02,-1.53091550e-01,9.45783705e-02,-7.88172245e-01,3.63510132e-01,-1.04770593e-01,-2.51155477e-02,-5.58734179e-01,-6.37229145e-01,3.46056968e-01,3.53124797e-01,-3.31066549e-01],[-1.51214242e+00,-9.04925242e-02,3.43349837e-02,-3.10094617e-02,-8.54466736e-01,9.34958179e-03,6.22804821e-01,1.34583890e-01,3.76446098e-01,-3.06857526e-02,9.35085043e-02,-6.75231993e-01,-4.52530384e-01,-1.91684082e-01,8.74387920e-01,-3.30610394e-01,-1.13613360e-01,-2.23034203e-01,8.18468332e-02,3.37399334e-01,-1.95829868e-01,-1.18460387e-01,3.31629902e-01,-5.63212454e-01,-7.77209103e-01,-4.39065605e-01,2.39879042e-01,-5.73004223e-02,-2.17636630e-01,5.54200634e-02,-3.38203281e-01,5.48899293e-01,-1.82235837e-01,-3.86965692e-01,-9.68845636e-02,-1.75358132e-01,4.49970841e-01,-7.09927231e-02,-8.73975083e-02,-3.48293811e-01,1.37239054e-01,6.50854051e-01,-1.09648776e+00,-5.53656220e-01,1.34751603e-01,4.52727616e-01,4.94476974e-01,4.94245589e-01,5.92882514e-01,5.68081513e-02,-3.35056372e-02,-3.44783634e-01,6.04499102e-01,3.89079005e-01,-6.86136857e-02,2.05021039e-01,6.17503822e-01,3.99755806e-01,4.83651459e-01,8.40284005e-02,-1.52307749e+00,4.01912570e-01,-5.97219355e-02,-4.83337730e-01,6.11069858e-01,6.64141715e-01,9.24657658e-02,6.35722056e-02,-8.64640355e-01,5.39445579e-02,5.23710847e-01,1.06765516e-03,-5.31464331e-02,3.09157819e-01,3.76388133e-01,-2.51754194e-01,2.80713290e-01,-3.21594208e-01,6.72466904e-02,4.90815461e-01,1.41604483e-01,-5.72821736e-01,-4.27154481e-01,-8.72002095e-02,-6.53326333e-01,1.59401596e-01,-2.48207912e-01,5.84204853e-01,-6.75907969e-01,-2.41417944e-01,1.32587239e-01,-5.10231078e-01,-1.38109118e-01,2.60791600e-01,7.44843960e-01,4.58059281e-01,-1.35110903e+00,1.77130252e-01,9.09532428e-01,4.98694360e-01,1.30286917e-01,-9.94639218e-01,-1.68989941e-01,1.04600221e-01,-5.92841268e-01,2.96895057e-01,-2.89252967e-01,-4.10633206e-01,3.38955462e-01,-2.01416742e-02,4.02447134e-01,-7.96317011e-02,1.06341970e+00,-6.85860515e-02,-7.30529249e-01,3.25979084e-01,6.06320389e-02,2.86655784e-01,-5.92438996e-01,-5.64232282e-02,2.93031424e-01,-4.02038395e-01,1.10386714e-01,2.83298403e-01,3.84425193e-01,-4.20072943e-01,3.63002092e-01,-4.22460921e-02,1.01963818e+00,-5.99131227e-01,3.03402930e-01,-1.33389518e-01,-5.70028424e-01,-1.63462654e-01,1.82333991e-01,1.88884772e-02,3.10962915e-01,4.37153339e-01,-1.06972492e+00,-7.38054276e-01,-3.65533054e-01,-6.84794426e-01,-5.88741899e-01,-4.87398863e-01,1.71227530e-01,5.63024506e-02,1.12804631e-02,-9.60451961e-01,-6.38031185e-01,5.18068254e-01,-4.55105722e-01,-2.54152030e-01,1.58398092e-01,-2.42026836e-01,2.13457197e-01,3.37402046e-01,-8.11240792e-01,-5.10384679e-01,-3.23240280e-01,6.78606378e-03,6.70450687e-01,2.36784071e-01,2.92183667e-01,1.54829040e-01,-2.82931298e-01,1.70756415e-01,-6.63495719e-01,-1.54054135e-01,-3.06861848e-01,3.84574741e-01,5.80521703e-01,1.07881375e-01,2.72290200e-01,-4.61985648e-01,3.92330766e-01,-3.43096815e-03,2.76843727e-01,6.80580795e-01,-2.83919036e-01,6.25746787e-01,-3.84000763e-02,-2.39067093e-01,1.28347978e-01,1.76458627e-01,-2.19890475e-01,3.79850268e-01,-4.09368664e-01,7.37610281e-01,7.96416998e-02,3.40901881e-01,-3.98178577e-01,-3.57051164e-01,2.58947104e-01,4.59974289e-01,-4.59276438e-01,-3.18441749e-01,-4.15129066e-01,4.93679166e-01,-7.17121601e-01,-7.66108871e-01,-3.20805281e-01,-3.89327496e-01,-2.94993877e-01,5.04008234e-01,-2.88890511e-01,-2.23947596e-03,-3.06600988e-01,-7.69994706e-02,-3.54305506e-02,3.57598513e-02,5.89347668e-02,-8.60639930e-01,4.15098548e-01,-2.71584630e-01,-1.18945949e-01,-4.24941480e-01,-4.97734584e-02,-6.13574730e-03,-2.84892678e-01,-5.71677089e-01,3.48783255e-01,-3.37432474e-01,-9.33071494e-01,-7.87689626e-01,-4.12068635e-01,-9.41796064e-01,-2.65520271e-02,-1.25262827e-01,-4.73450541e-01,1.20460704e-01,-2.58906901e-01,6.03507221e-01,-8.69944513e-01,3.31792563e-01,5.92063293e-02,-2.30112091e-01,2.30257958e-01,-1.33753967e+00,7.46930167e-02,5.66187538e-02],[-0.20078108,0.22533426,-0.36364472,-0.9757074,-0.13758108,-0.11964672,0.14948408,-0.33626482,-0.3333954,-0.10562959,0.4040446,0.04746432,0.5285222,0.504251,1.2029309,0.5902482,-0.05076688,0.00389656,-0.05979482,-0.08725154,-1.1867719,0.51583505,0.18108727,0.06506676,-0.22226098,0.38566607,0.2829473,0.2565636,-0.62792605,-0.1917304,-0.21587153,0.2961585,-0.10203194,-0.28933236,-0.20620866,-0.43375084,-0.25632307,-0.10494073,-0.81391823,-0.5104663,-0.39460915,-0.50486314,-0.07651348,0.4321516,0.11277548,-0.12492061,0.09071638,0.05723809,-0.73112214,0.00151823,-0.67112035,0.32769006,0.04034228,0.76195186,-1.0050625,-0.09541111,-0.37083468,0.49867278,-0.7195877,0.7306738,-0.6347043,0.8254477,0.38831902,0.20813008,0.4261452,0.02397975,-0.42202097,0.13874489,0.12395293,0.23125672,0.21488388,-0.2811733,-0.77453893,0.21812038,-0.17485271,0.08108754,-0.20049106,-0.04838685,-0.11464927,-0.6832779,-0.07117452,0.18637078,0.5286139,0.64678997,-0.02728922,-0.26828578,0.6397908,-0.0167776,-0.2891698,0.34238702,-0.2853441,0.8082939,0.15612884,-0.07812928,-0.23470843,-0.21030524,0.10113859,0.15028115,0.5730223,-0.62626785,-0.3320637,1.0800155,-0.15437536,0.06441247,-0.9861085,-0.7261967,0.5920037,0.47801763,0.37712473,0.37714434,0.4854574,0.5693471,-0.08450921,0.49534875,0.46586624,-0.21984652,0.10302423,0.42755142,0.07778049,-0.8119206,-0.23347145,-0.01281973,-0.2383521,-0.08921731,-0.2568462,-0.9020558,-0.06537967,0.51134074,0.35819355,0.19856961,0.48775023,-0.44051862,0.03741527,-0.6352103,-0.358531,0.31594488,0.7244167,-0.30222103,-0.30500478,-0.15959711,0.44588265,-0.23227453,-0.8806135,0.312559,0.23139141,1.0306859,-0.5043165,0.02901264,-0.37556592,-0.04328266,-0.43003124,-0.15493546,0.26767477,-0.3886135,0.41371402,-0.57345605,0.0516627,-0.05937927,0.23841861,-0.08966246,0.59332734,-0.08862029,0.14700605,0.15162414,0.59272474,0.43360144,1.0798757,-0.6587499,-0.0859787,0.07410391,0.5226149,-0.08572339,-0.8414845,-0.22115156,-0.05724639,-0.17124192,-0.57845455,-0.18867283,-0.2796527,0.0782975,0.05123539,0.09832998,-0.45679453,-0.56956345,-0.14256018,0.23837723,-0.08269215,-0.71502674,-0.2614112,0.4639847,0.33431903,-1.456555,-0.29424703,0.2274513,0.358802,0.057416,-0.10339779,-0.17315406,-0.10573076,-0.13464099,-0.60086626,-1.0767004,0.47684026,0.17439383,0.75706434,0.365016,0.28808072,0.07612307,0.03516015,0.1407304,-0.52474105,-0.01431718,0.0073433,0.08798484,-0.6923621,-0.2420583,-0.32238588,0.41887903,0.45951873,-0.7712289,1.0321026,-0.3566561,-0.01469571,0.20153596,-0.40288192,0.14330858,-0.04736254,-0.10744917,-0.22502458,0.07793895,-0.25161198,-0.02352532,0.44933608,0.18923381,-0.24000981,-0.15864012,0.48935485,-1.1106246,0.74218166,0.2554432],[-0.06488599,0.2723192,-0.5567748,-0.30302492,0.67704093,0.08617077,0.4288497,0.20771301,-0.8340965,-0.11714417,0.08593322,-0.87307036,0.1728235,0.04566931,-0.26766196,0.49168152,0.49625823,-0.21386349,0.44133562,0.03929631,-0.22591452,0.68059754,-0.06963122,-0.2428698,0.15640676,0.65021354,0.69665855,-0.08581916,0.2645626,0.25951034,0.13571654,-0.09974661,0.21792217,-0.05902296,0.12572405,0.00497331,0.11090454,-0.81455654,0.35958308,-0.52995974,-0.12838599,0.46305844,0.5253301,-0.14423034,-0.38732108,0.49579635,-0.36244756,0.06137522,0.4165109,-0.7050326,-0.16207244,-0.5799114,-0.46376377,0.17344572,-0.20813686,-0.5910748,0.1516674,-0.10084435,0.2497377,0.37345824,-1.3271766,-0.47104803,0.40234804,0.21565735,-0.7574194,-0.41707876,0.32389277,-0.7812428,-0.22585393,0.67076284,0.00305064,-0.17491378,0.84318656,0.42871857,-0.04988671,0.2244829,0.2557695,-0.28146836,0.26076505,-0.06094579,0.94984674,-0.32587174,-0.3498737,-1.126571,0.6276627,0.23035051,-0.62717086,-0.00286172,0.3712382,-0.03809109,0.19273639,-0.01218705,-0.15643372,-0.2376217,0.37385148,-0.92706317,-0.01420997,-0.5882955,-0.08899888,0.09292275,0.5964959,-0.36761257,0.8373939,0.1378876,0.6864888,0.02135257,-1.4427671,-0.27717412,0.37789986,0.07042711,-0.81844056,0.289426,-0.4131253,-0.8018496,0.29943973,-0.4236383,0.20021154,-0.4013353,-0.45121437,0.69761395,-0.47511852,0.15896225,0.00505819,-0.15348718,-0.34529847,-0.17705163,0.29076415,0.61221546,0.2903855,0.261666,0.20904383,0.1318263,0.39599988,-0.3984495,-0.0250873,-0.01786288,0.2859385,-0.4307507,-0.579678,-0.09303512,-0.66310394,-0.46207398,-1.272954,0.11661788,-0.69696957,-0.08493436,-0.54962146,-0.03651948,0.3219852,0.32509875,0.11570781,-0.10445736,-0.6153903,0.05517009,0.02380925,-0.37841552,0.14600684,-0.80303586,-0.9200605,0.37389278,-0.7009278,0.24815635,-0.33424598,-0.19059058,-1.041966,0.49946377,0.75319636,-0.12620205,-0.21396574,-0.41764542,-0.2812552,0.06898516,-0.45606664,0.36889336,-0.7489699,-0.12262946,0.5386853,0.05266838,0.03981014,0.07675325,-0.02696864,0.4632765,0.41450202,-0.9655414,-0.8118372,0.31265604,0.65312403,-0.36236537,0.28749716,0.71674085,0.5162457,-0.5028821,0.08702556,-0.05361639,0.19753602,-0.00682396,-0.68913984,-0.05847797,0.3682675,-0.41077036,-0.11021607,-0.13005571,0.25642064,0.20197865,-0.24228258,0.00843715,-0.578375,-0.01742559,-0.15185189,0.7026315,0.5523086,-0.39001006,-0.23449488,-0.07015252,0.04691476,0.34775883,-0.36283708,0.23259853,-0.05813894,-0.25228545,0.6187358,-0.9047584,-0.47071874,1.2095083,-0.05146547,-0.40301168,0.32647276,-0.18959273,0.553171,-0.7086991,0.17446066,0.23184869,-0.14195399,-0.38593712,0.2758626,-0.22872177,0.3394612,-0.5048133,-0.21658751,0.35042927],[0.23043083,0.05906814,0.03424101,0.26987433,0.07489629,-0.26916015,-0.80293757,-0.09247515,-0.72493285,0.3709348,-0.26760107,0.12864436,-0.12030329,-0.3703414,-0.1266692,-0.2434843,-0.64000416,-0.5098649,-0.06204452,-0.37503752,0.36460027,-0.29390118,-0.5116935,-0.5894329,-0.2045817,0.42878172,-0.4799327,0.7796561,0.14714836,-0.19478072,-0.28562343,-0.4958573,-0.06837147,-0.11424135,-0.7154569,-0.45962074,0.40470654,0.12614183,0.741915,0.68368065,0.48890856,-0.54164445,-0.00564975,0.11312202,0.10078134,0.1725349,1.0252141,0.37514663,-0.26213774,-0.12260511,-0.28035372,0.61865664,-0.58663,0.47232038,0.31728512,-0.26719552,0.1858038,0.05264913,-0.17768973,-0.00915186,0.00722711,-0.27501157,0.0901212,-0.10041045,0.86528695,-0.30122858,-0.2153327,-0.17246181,0.03744113,0.5563626,-0.7391246,0.78032225,-0.06250861,0.65871406,0.17172138,-0.21481203,-0.08012262,-0.76112336,-0.7612971,-0.21069537,0.23208143,-0.7294191,-0.45878136,0.12957275,0.00811148,0.19450572,0.04693819,-0.22626127,0.42880192,-0.3677967,-0.69222414,-0.36060917,-0.49493906,-0.32351744,-0.6849226,0.963321,-0.41773796,-0.05779459,0.31613755,-0.18671149,0.14661695,-0.6735855,-0.21111897,-0.1328828,-0.5950658,-0.4143787,0.5932818,-0.7360404,-0.38422483,0.719626,0.50270814,-0.3173565,0.615061,-0.42671233,0.4089649,-0.13594776,-0.23714373,0.12955269,-0.53270894,-0.17792584,0.37144712,0.24182369,-0.55215544,0.3828193,0.32691067,0.03398237,-0.82501775,0.03536182,0.18180978,0.2498829,-0.5362297,-0.38226354,0.06244337,0.40336868,0.39405322,-0.05198174,0.3127493,0.13276146,-0.11399239,-0.05168824,-0.06100919,1.0774152,0.0474757,-0.6899572,-0.7350547,-0.26167527,-0.0915847,-0.14438437,0.737412,-0.38031977,0.43411905,-0.8240148,0.41754034,-0.09406362,-0.0509904,0.5887481,0.31694183,-0.35441434,-0.16326548,0.5194791,0.657257,0.5012017,-0.07644253,-0.24409613,-0.02618274,-0.16937204,-0.38837278,0.30880776,-0.27665338,0.21204157,0.33579355,0.93749535,-0.3474701,0.504212,-0.13471057,-0.2206004,-0.3505814,0.16224554,-0.02441031,-0.24353735,0.96103567,0.8866572,-0.4595183,0.18900892,0.2041661,-0.3556425,-0.9205012,0.27972624,0.09570196,0.2785476,-0.01661253,0.63306814,0.07438757,-0.37688425,0.0783427,0.8624206,-0.48421624,0.35379493,0.716356,-0.45623156,-0.15314664,0.0013804,0.07550346,0.32885176,0.12975393,0.43451256,-0.57683194,0.33255965,1.2656019,0.15543935,0.14989553,-0.08974813,-0.12803984,0.5452251,-0.07378991,-0.36356822,-0.41124657,0.12404147,0.03608922,0.47487947,0.3809242,-0.78913605,0.44880286,0.15155783,-0.707368,-0.03112323,0.0483585,0.20356752,-0.8434144,1.0090884,0.07011557,0.17662647,-0.66483444,0.29919127,0.20908336,-0.62603897,0.16089253,-0.1679785,-0.14564866,0.7472264],[-4.60556269e-01,-4.76418138e-01,4.41021860e-01,-8.14666092e-01,-5.91020942e-01,6.02010012e-01,-3.55699748e-01,-7.24853814e-01,-6.41903758e-01,-6.77191466e-02,-3.50639045e-01,-8.66496339e-02,-4.77799803e-01,9.06947926e-02,-1.41365480e+00,1.78715199e-01,5.42246044e-01,3.46975327e-02,2.70555109e-01,-9.00534540e-02,-2.47500792e-01,2.42365062e-01,-3.61900069e-02,4.90184098e-01,1.19633026e-01,-6.19833827e-01,-5.11755109e-01,-1.12666659e-01,-1.11137740e-01,1.55615434e-01,1.56500079e-02,-2.40057945e-01,7.65935600e-01,-6.12847745e-01,-4.62835848e-01,-1.88455373e-01,-7.75294662e-01,4.22639817e-01,-2.99032569e-01,-3.52212131e-01,-2.74963945e-01,2.16654211e-01,3.65623981e-01,-1.23441875e+00,-3.76508743e-01,-2.74391472e-01,-2.41330922e-01,-2.55744338e-01,-3.41355830e-01,1.39978558e-01,5.76180756e-01,-7.91047990e-01,-3.82858187e-01,-1.05033123e+00,-1.19430698e-01,-2.23280221e-01,6.16281390e-01,-8.61421704e-01,-1.09290712e-01,-1.07465878e-01,-5.21292984e-01,1.17731847e-01,-3.77011597e-02,1.00098407e+00,4.37843800e-03,-1.15481205e-01,2.49266833e-01,4.19020355e-01,2.22956076e-01,-8.00864279e-01,2.70880610e-01,1.49119601e-01,-1.02831036e-01,1.62309721e-01,-1.81471959e-01,5.75391464e-02,6.12715662e-01,-4.89138663e-01,2.49055877e-01,1.45704344e-01,5.91419339e-01,3.86047930e-01,2.86361426e-02,1.88873589e-01,1.90535054e-01,5.87174118e-01,6.45204186e-01,2.50564694e-01,3.51766825e-01,3.29924405e-01,1.65507168e-01,-3.08129787e-01,-1.54666334e-01,1.00228071e-01,-3.16406846e-01,2.10282683e-01,5.46304047e-01,8.43000710e-02,-7.75502399e-02,3.04566175e-01,-4.19924945e-01,-6.00643635e-01,6.76035509e-02,3.49885345e-01,-1.97137639e-01,4.57458466e-01,3.31242889e-01,1.33390799e-02,1.38761953e-01,-1.91646367e-01,-1.10421404e-01,7.25379437e-02,4.04739559e-01,-9.62113440e-02,-2.97437102e-01,-4.19873931e-02,-1.27344072e-01,-2.26836070e-01,2.66718030e-01,2.31143251e-01,3.46766934e-02,1.49468690e-01,8.76185223e-02,-8.03125978e-01,3.10876984e-02,-1.20940723e-01,-1.09337485e+00,7.34464228e-02,-6.46406472e-01,1.65077031e-01,5.93440413e-01,-1.73518702e-01,-3.87279168e-02,2.25097224e-01,-3.55506241e-02,-6.15392663e-02,-3.04557890e-01,4.02572811e-01,-1.77937701e-01,7.14234591e-01,-7.03060210e-01,-7.60799944e-01,7.59969875e-02,4.30490434e-01,-3.98826562e-02,-1.83526456e-01,-2.70003915e-01,-3.47462893e-01,7.76010573e-01,2.50927240e-01,-6.71080709e-01,2.18593016e-01,-1.41618028e-01,-6.64262706e-03,7.54347503e-01,-6.27148271e-01,3.64615589e-01,-3.38811398e-01,8.59912157e-01,8.58870685e-01,-6.73369527e-01,1.60313606e-01,3.80272269e-01,-2.87133485e-01,-8.47597957e-01,4.16299999e-02,2.01850653e-01,-3.09340417e-01,-9.46168453e-02,7.79065937e-02,-5.87224305e-01,4.74992514e-01,7.96217322e-01,-4.73453291e-03,-2.88648754e-01,4.97135431e-01,-9.15454328e-02,2.29291730e-02,1.44347996e-01,1.85745843e-02,-2.53700107e-01,-3.19495440e-01,-2.67235130e-01,8.01337063e-02,-2.23772556e-01,-5.22478104e-01,-1.31710514e-01,4.46874946e-01,-3.69932860e-01,-7.51244664e-01,-7.41641164e-01,-7.20656395e-01,3.05624977e-02,-5.94770648e-02,-1.16025582e-01,1.88901514e-01,-6.69004083e-01,5.51590860e-01,1.87169760e-01,3.69914681e-01,-2.78105617e-01,9.50809181e-01,-4.57101405e-01,1.70650221e-02,-3.46951216e-01,1.76939622e-01,-2.72589535e-01,3.35650444e-01,-1.79000869e-01,-7.24715650e-01,2.55921930e-01,1.08609664e+00,-6.79950953e-01,4.52375233e-01,-2.35044211e-01,1.52108982e-01,3.63228321e-01,-5.09397069e-05,3.81098628e-01,6.70957804e-01,7.00956881e-01,1.58340573e-01,4.91116792e-02,-7.22831190e-01,6.39889836e-02,-6.08465374e-01,3.00901942e-02,1.75111815e-01,1.57727495e-01,-1.28218517e-01,-5.44312000e-01,-1.47498444e-01,7.97971785e-01,3.11205298e-01,-4.31366861e-01,5.41452229e-01,7.37068132e-02,-8.23230565e-01,4.12423015e-01,-4.17399019e-01],[-0.13666733,-0.1763568,-0.19624054,-0.4112684,0.12232565,-0.18155335,-0.36455306,0.4878159,-0.05521265,-0.42750412,0.64770854,-0.09405057,0.3349142,0.165339,-0.7530596,0.37779495,-0.6339581,-0.42738885,0.6531821,0.37537703,-0.03077185,0.34182388,-0.22472927,0.05085537,0.24736878,-0.07396398,0.048583,0.62497276,-0.11592179,0.46461096,-0.16985163,-0.07754727,-0.12447067,-0.05049438,0.63409233,0.31912923,-0.45783818,-0.08004619,-1.184342,-0.8146132,0.12679908,0.20475066,-0.6917016,0.09976476,0.4634568,0.23135167,0.24483731,0.19351031,-0.16984688,-0.10972843,-0.44685718,-1.4988681,-0.22966096,0.49019974,0.17107026,-0.2959445,-0.6282816,-0.24541228,0.63609815,0.16988578,0.05742974,-0.56340235,-0.5589423,-0.26065832,-0.00880521,0.10183626,0.14196944,-0.17067121,-0.93136907,-0.8384521,0.14490207,0.55151165,-0.28474894,0.0802274,0.14408942,-0.03907472,-0.2725342,-0.24840292,0.17416511,-0.14564152,-0.5482977,-0.56471413,-0.20489013,-0.01693171,0.57780594,0.6625152,-0.09737021,-0.6202444,0.24737424,-0.50406504,-0.07186259,-0.08459027,-0.7597132,-1.1247892,-0.16167754,0.19472279,-0.3162933,0.15328178,-0.08276708,-0.51070833,0.00417606,-0.02825709,-0.15245889,0.27216673,-0.09918049,0.31625152,0.10019365,0.09225106,-0.31749976,-0.55381644,0.05491085,0.49400786,-0.7444231,0.25119448,0.06474758,-0.2370388,0.14831989,-0.06029608,0.4248736,0.28519621,0.62120295,0.08027684,-0.15501802,0.07762906,0.10843059,0.1810009,-0.411707,-0.0394072,-0.23008263,-0.01536453,-0.10029704,-0.38920653,-1.0166996,-0.42605567,-0.22795002,0.04635358,-0.07117444,0.35011977,-0.11684935,0.07470103,0.10937217,0.04316651,-0.06088421,-0.09777047,-0.43447262,-0.49061748,0.41439223,-0.55566555,0.3134731,-0.25036556,-0.40913576,0.30055487,-0.19357736,0.00198808,-0.5881536,-0.02478198,0.95396256,-0.4898564,-0.50531715,-0.06579829,0.07680532,0.7469934,-0.19192308,-0.17485633,0.11752909,0.2963328,-0.25951517,-0.8807397,0.05313056,0.2502542,0.1450047,-0.14347863,0.2768185,-0.27418798,-0.6607348,-0.5324127,0.2092623,0.43553552,-0.02551161,-0.6890627,0.22535063,-0.28284803,0.16510521,0.23553863,-0.63318795,-0.4184695,0.5588175,-0.05427364,0.5087594,0.5546742,0.3732712,-0.19035569,-0.19813703,-0.47262308,-0.01194154,0.18595001,-0.04410509,0.31060228,-0.2403217,-0.17134358,0.16061734,-0.4624784,-0.19202796,0.22128926,-0.46839046,-0.05493629,0.48409784,0.5223724,-0.29865482,-0.2269131,-0.8276539,-1.1387677,-0.14972144,0.43951997,-0.13345328,-0.31691656,-0.05349773,-0.56734943,0.02889421,0.4700494,-0.10417557,0.19190593,0.24290748,-0.04224665,-0.18842515,-0.314858,-0.39600775,-0.13957384,0.2019381,-0.39324164,0.64317894,0.1974604,0.15917711,0.67217,0.1490783,0.5168761,0.15925491,0.1196118,0.06454688,-0.27774364],[0.28518948,-0.13997601,-0.5556737,-0.04169525,-0.4290255,-0.38601238,-0.26724187,0.5590105,-0.10568724,-0.35473615,-0.41717026,-0.13898893,-0.46908528,-0.38653794,-0.85160744,1.1219306,0.20181896,-0.16410312,0.21059646,0.03212613,0.55940473,0.36968145,-0.3721874,0.4646146,0.13253291,-1.0026734,0.28006747,-0.05986625,-0.9621412,-0.08339182,0.04610351,-0.97930515,0.89615774,0.42997295,-0.09211554,-0.27109265,-0.13400462,0.10549568,-0.696152,-0.2580794,-0.2182579,0.4325452,-0.24234511,-0.35840642,-0.8378016,-0.16632926,-0.79030246,-0.17635542,-0.03329151,0.02400197,-0.4044793,0.7697253,0.6847476,0.14756228,0.22840151,0.5917247,-0.19160013,0.1976704,0.00299746,0.00810256,0.22467378,-0.7368519,0.02175383,0.27360344,0.4444114,-0.1802902,-0.22928818,-0.0988891,0.63900924,-0.24284792,-0.03690095,0.4584271,0.22036052,-0.6583239,0.0261431,-0.40477654,0.06488252,1.1946772,0.52841574,-0.5904566,-0.9307232,0.04548438,-0.20389047,0.04549177,0.403355,-0.07419919,0.14176144,-0.23832779,-0.74980193,0.43680465,1.0384319,0.4092359,-0.3926028,0.37901065,-0.44184965,-0.2515458,0.8497127,0.15571964,-0.5317493,-0.1261214,-0.7487016,-0.6250402,0.04405375,-0.02469361,-0.04940774,-0.31688696,0.45927164,-0.33307183,-0.11606286,-0.08885399,0.06313904,-0.35124245,0.33998948,0.03200768,-0.2820687,-0.133917,-0.04375549,-0.62433606,-0.53799,-0.4477209,0.08060294,0.22371155,0.31586242,-0.79798704,0.3360504,-0.2721379,-0.23732142,-0.10862433,1.0114722,-0.50278777,-0.02639589,-0.02403699,0.0174925,-0.36008802,-0.14972576,0.2086053,0.5566123,-0.40467718,0.43621406,-0.31540063,0.29862702,-0.25498852,-0.5726091,0.02833777,0.32155004,-0.79250985,0.14185296,-0.21760066,-0.9772647,0.94512886,0.38399515,0.10882018,-0.30262753,0.34874073,-0.42187515,-0.3072515,0.15214117,0.54100275,-0.07661238,0.27124184,0.4926746,-0.01097856,0.2403205,-0.19304213,-0.12211669,-0.18039674,-0.05341323,-0.29510573,0.7716716,-0.41356385,-0.09851451,0.08862106,-1.2080615,0.43369946,0.5639641,0.08142772,-0.6241265,-0.39350158,-0.36529836,0.39691105,0.31725025,-0.17088577,0.19277495,0.1986205,0.06950497,0.8395866,-0.13649662,0.28484324,-0.29386678,0.07089487,0.71944964,0.16465476,0.9360646,-0.38914078,-0.53558654,0.21210998,-0.25662386,-0.07536432,-0.50000536,-0.24963008,-0.35334733,0.25703266,0.11659896,0.21222298,-0.30023986,0.6443201,-0.08124709,-0.2955156,0.48266008,-0.5721394,-0.4314905,0.03271056,-0.01003959,0.18659955,-0.16029982,-0.17753658,-0.17227915,0.2818045,0.37895733,-0.4834516,-0.77365106,0.5161814,-0.778941,0.8037131,0.6937076,-0.4727712,0.43439022,-0.22564039,-0.04563778,0.09029891,0.36015683,0.47572434,0.34703264,-0.36813998,-0.521424,-0.62056893,-0.00227982,0.09429298,0.02319865,-0.40284175],[0.6237201,-0.75275826,0.19984698,-0.11120279,-0.3247145,-0.70461285,0.1178989,0.4289228,0.31078506,0.10348504,-0.31470874,0.31090698,-0.26246816,0.4689369,0.23021625,-0.6289094,-0.04927088,-0.31307718,0.277006,-0.35300395,0.1688917,-0.23158415,0.14930381,-0.87508047,-0.24899898,0.07384057,0.01877206,0.55881816,0.25413713,0.3132793,0.43576697,0.3533915,0.340238,-0.42893192,-0.12626404,-0.4456382,-0.31562415,-0.07110627,-0.29801017,-0.42308208,0.93618613,0.13087323,-0.69869703,-0.50689036,0.18827288,-0.2824274,-0.67615986,0.791616,-0.62587494,0.57741314,0.55460685,0.19877353,-0.33541748,0.24155535,0.2345967,0.5761845,0.1307927,-0.36653244,0.2846814,0.47082865,-0.24106744,-0.06873049,-0.26085788,0.14770724,-0.14202052,-0.3962691,0.0618475,-0.8714202,-0.02772547,0.12634249,0.1900272,-0.52348226,0.45001483,0.14148149,-0.17751935,-0.22581784,-0.1446356,-0.26120535,0.5586675,-0.38133973,0.07077669,-0.37964502,-0.21574071,0.06660645,-0.2001702,-0.40085587,-0.06522175,-0.8739349,0.8300144,-0.41216362,0.38491553,-0.3433968,0.41279435,0.1746553,1.2108657,0.45831394,0.2305231,-0.5422702,0.07124805,-0.20955192,0.32806608,-0.45523274,-0.04927513,-0.4950995,0.00277077,0.11901433,-0.32696626,0.37180367,-0.09079738,0.576916,0.13715816,0.08382595,-0.50593024,0.41288662,0.00135702,0.27894318,-1.074469,-0.3937553,-0.13791387,-0.35751915,0.75449735,-0.35429004,0.2317916,-0.07573484,0.6025068,-0.44986308,-0.15656036,-0.28981343,-0.63313293,0.31224325,0.31829846,0.39491364,-0.01957931,0.12670626,0.05341045,-0.16855751,-0.11079875,-0.02697297,0.12649143,0.20874721,-0.04355343,-0.70959574,0.38001114,-0.16487353,-0.25353494,-0.2575903,0.22916424,0.1064438,-0.06450401,0.35914746,0.06861562,-0.23835701,-0.49049693,-0.03186518,0.68993384,0.58815485,0.15671939,-0.00523824,-0.40624177,0.36525336,-0.9828212,-0.5163228,-1.0137249,0.3618795,0.10284937,-0.28486127,-0.02374595,0.166013,-0.12605633,-0.7499929,0.6022741,-1.1390691,1.088239,0.8725295,-0.5400409,-1.1077329,0.10600133,0.01973992,1.1284,0.26328993,0.7206746,-0.9784361,-0.00310059,-0.6506535,0.01351794,0.01827019,0.15679926,-0.06369413,-0.12496529,-0.13987507,-0.44937214,0.52065283,0.4446041,0.0728088,-0.86969167,-0.50383705,0.07996204,-0.8709308,0.345629,-0.80015993,-0.18857564,-0.37166622,0.3633294,0.76632655,0.22776678,0.5692733,0.01004294,0.51162213,-0.30533713,0.25895882,0.02280508,-0.6011364,0.16440001,-0.3891718,-0.31978792,-0.2133045,0.18187334,0.27049717,-0.33933577,-0.4074831,-0.05997669,-0.2394722,-0.21969742,-0.19808306,-0.41793314,0.5520362,-0.19069017,0.6475025,-0.74507064,0.47862172,-0.34434542,-0.7446681,-0.01174131,0.5651004,0.14411163,-0.28194177,0.2897525,0.09340025,-0.5751687,0.55331075],[5.74478554e-03,-3.69619638e-01,4.71915424e-01,4.91213888e-01,-1.14326077e-02,-1.21325374e-01,-3.38591367e-01,-3.49446714e-01,2.31416479e-01,7.78887868e-02,-2.22811937e-01,-4.80526298e-01,4.25695539e-01,-5.33596933e-01,4.77756336e-02,-2.17232421e-01,-9.72551286e-01,-3.31713587e-01,1.32258403e+00,9.79872644e-01,2.23692715e-01,-1.46892490e-02,6.79640174e-02,-3.21511716e-01,8.68147798e-03,1.79590702e-01,2.76123993e-02,3.82677585e-01,2.63330042e-01,-5.90738773e-01,-4.33857381e-01,-5.74276507e-01,3.57283682e-01,9.17323008e-02,-3.63085389e-01,-2.21903831e-01,2.59396374e-01,5.23093700e-01,-3.03367198e-01,6.89732254e-01,-3.25409204e-01,-3.47457916e-01,-7.14840293e-01,2.74634272e-01,-4.69574988e-01,-8.16228747e-01,8.08524668e-01,4.38341022e-01,4.17290717e-01,8.30120891e-02,-4.59395021e-01,-4.40444648e-01,3.90085876e-01,3.50804441e-02,-1.83391616e-01,-2.35992596e-02,4.18542065e-02,-4.11410421e-01,-3.12449783e-01,-2.25578189e-01,-2.66421556e-01,-2.96639800e-01,9.19735909e-01,7.32076824e-01,3.34378511e-01,4.51184481e-01,-5.92496768e-02,5.72633594e-02,1.10588670e-01,2.99814641e-01,-7.56495297e-01,-3.69126201e-01,-1.31715098e-02,1.57038886e-02,-2.64699847e-01,-4.46541160e-01,1.14199921e-01,-3.79324526e-01,-2.44124010e-01,1.19294524e-01,2.40594801e-02,-6.02600686e-02,4.66357410e-01,1.18136990e+00,3.80196422e-01,1.99353889e-01,-8.53694677e-01,-1.04214561e+00,-2.52515972e-01,1.68978386e-02,2.09700570e-01,-3.15057695e-01,-5.34522831e-01,-1.16276145e+00,2.87848383e-01,1.71715990e-01,-2.17272878e-01,1.11270033e-01,-7.88676500e-01,6.90235049e-02,-2.88120985e-01,3.12532008e-01,5.38671426e-02,4.80885088e-01,-1.38576970e-01,4.58346261e-03,2.24781424e-01,-5.30894935e-01,3.86826634e-01,2.91542616e-02,-6.88214824e-02,3.34064424e-01,-1.54963788e-02,-6.59376502e-01,-5.35386086e-01,1.28482953e-01,1.39784262e-01,1.99899703e-01,-1.07413612e-01,-8.10450837e-02,3.00535917e-01,-9.78765786e-02,2.98207909e-01,-6.41256645e-02,1.65153980e-01,-7.52379894e-02,3.08339119e-01,3.01299810e-01,-3.11972558e-01,-3.19671631e-01,1.54889539e-01,1.12885386e-01,3.16249698e-01,-1.44672602e-01,-8.19071293e-01,-1.04451454e+00,-1.24053530e-01,-3.04102227e-02,-2.05914691e-01,-4.86359060e-01,-9.27645788e-02,-3.32218222e-02,1.35013416e-01,2.47724265e-01,3.89664650e-01,-1.58471931e-02,-1.22577143e+00,-2.06160098e-01,1.95393279e-01,2.94612646e-01,8.91597867e-02,3.60999793e-01,7.59267509e-02,2.16015741e-01,2.87861139e-01,3.03983331e-01,2.08325773e-01,7.98655078e-02,-7.94873089e-02,-4.40945446e-01,-5.17264545e-01,-5.34292720e-02,-2.02607438e-01,6.39142096e-01,-1.20910011e-01,1.02635277e-02,9.13007796e-01,3.40939537e-02,-2.58356929e-01,-1.85186416e-01,-2.68043160e-01,-1.57648567e-02,4.52056259e-01,-2.24930495e-01,-3.77833575e-01,1.42377585e-01,-6.55776709e-02,2.14385569e-01,5.18618077e-02,2.08260253e-01,-5.46717107e-01,-4.87543941e-01,-4.80972379e-02,4.43258919e-02,8.18535924e-01,1.12741005e+00,-3.71671468e-01,-5.31597912e-01,3.89794372e-02,7.72178024e-02,8.69894922e-02,6.29029125e-02,5.99362493e-01,2.16575533e-01,-4.49524581e-01,3.21092039e-01,3.89676392e-02,7.72498548e-02,8.10765661e-04,-4.34600294e-01,5.64919561e-02,5.02577126e-02,-2.97274709e-01,-1.82436004e-01,1.72372814e-03,3.86980087e-01,-4.24795747e-01,-2.87956893e-01,1.05242170e-01,-4.87104714e-01,-4.13026884e-02,-3.29981387e-01,-5.46738088e-01,-3.26600045e-01,4.44265693e-01,9.56336483e-02,-9.84457359e-02,9.05916691e-02,3.29578042e-01,1.00167088e-01,-3.70376885e-01,-4.55074012e-01,-1.86848119e-01,2.05678418e-01,7.89354816e-02,4.33695205e-02,-6.30750775e-01,-1.16086113e+00,-4.64179784e-01,-1.12030111e-01,2.17863858e-01,5.64349964e-02,-6.89290762e-01,-3.49972785e-01,3.51775318e-01,3.29290599e-01,-9.00221914e-02,2.53022164e-01,-2.59336174e-01,-3.31670523e-01],[-0.0581943,-0.14882301,-0.49881658,0.5421677,0.30623648,-0.07561509,-0.29535422,0.2085829,0.42784402,0.11082052,0.13482744,-0.12561066,0.77828556,-0.40851775,0.3828437,0.03378103,-0.33284026,0.05454691,0.3027297,-0.47928467,0.00712495,0.3305426,0.07745681,-0.27587292,0.7310382,-0.602926,0.49757898,-0.49370292,0.08754528,-0.46857,0.31966394,0.41145787,-0.39160463,-0.22938031,0.48574972,-0.43735158,-0.5418561,0.16946776,0.10243785,-0.2727029,-0.10891888,0.31431124,0.9444634,0.18582064,-0.2121559,-0.6946718,-0.21313931,0.4633618,0.18887584,-0.53745294,-0.46471027,0.17211457,0.32672623,-0.20559597,0.7092099,-0.48424208,-0.43620753,-0.7937568,-0.6032055,0.18402165,-1.1944456,0.29910415,0.29389694,0.17861041,0.6378072,0.5470886,-0.51218003,0.44670662,0.11476631,-0.32609117,0.06566043,-0.45219225,0.42450336,-0.27713525,-0.6011725,-0.11310918,0.5273976,-1.0368702,0.4077253,-0.7419169,-0.7248798,0.31851465,-0.16376467,-0.02504923,-0.57101846,-0.00802122,0.4044227,0.52867323,-0.44884107,0.11612139,-0.26012355,0.08606376,0.32837638,0.27801943,-0.30968472,0.30745813,0.25235242,-0.4472615,-0.45487842,0.64666384,-0.22454342,-0.18165748,-0.6293373,-0.03620462,0.22467732,0.26904655,-0.12665713,-0.30683294,0.47475567,-0.67925733,0.36866388,-0.19390948,0.28047708,0.72815657,-0.18430795,-0.868248,0.13165045,0.6605454,0.2576839,0.19366682,0.44475508,-0.4490028,-0.09061452,-1.1495463,-0.16972482,-0.16445069,-0.61365694,0.79260576,0.49218097,-0.04605736,-0.55976415,-0.11700374,-0.11627362,0.30372646,-0.5602656,-0.09228304,-0.5534971,-0.46757254,-0.06568494,-0.1550852,0.28032115,0.69035125,-0.17928867,0.3523524,-0.15833211,0.03086732,0.72147155,-1.2611076,-0.29367274,0.01666489,0.08969387,0.31323084,0.54096085,-0.21011214,-0.17901397,0.02975721,-0.10327806,0.2632476,0.43729192,-0.31897625,-0.39701137,0.16443048,0.75308245,-0.19150363,0.09656699,0.15922794,0.23093398,-0.1533462,0.08081009,-0.09080236,0.27752975,-0.5219917,-0.55340457,0.07876865,0.16677399,-0.2186065,-0.11758009,-0.03926573,-0.8821109,-0.3811288,0.21140257,0.6125788,0.56950957,0.06480707,0.8091343,-0.2601873,0.34764022,-0.14282855,-0.14537123,0.42064187,-0.583324,-0.95097727,0.4053855,-0.07616168,0.01804909,0.14071494,-0.31414318,-0.75098383,0.5764443,0.16433023,-0.85323787,-0.3296255,-0.32762358,0.69154996,-0.0115312,-0.493259,-0.1591338,0.13997702,-0.15796642,0.16124228,-0.04665714,-0.05027779,-0.04973526,0.38028598,-0.23128244,-0.21760777,-0.7203903,0.10890678,-0.44979042,0.6786052,-0.12188598,0.15853058,-0.39910498,0.3220335,0.19594091,0.23621301,-0.59046185,0.06135702,0.17388625,-0.69695055,0.11654996,-0.18188336,0.08976135,0.15095131,0.64439934,-0.13443276,-1.512214,-0.44961613,0.39781713,0.1930809]])
net.state_dict()['block3.0.bias'][:] = torch.tensor([-0.55591834,-0.3545469,0.65464157,-0.515388,-0.74371517,-0.28487813,-0.576413,-0.043503247,-0.06523518,-0.5175742,-0.40294793,-0.75252724,-0.5524272,-0.3122087,-0.10323482,-0.6365093,-0.09488646,-0.3110806,-0.23764034,-0.79092026,0.24668612,0.62291276,-0.37420508,-0.32181677,-0.73744196,-0.14300884,0.37413424,-0.0645985,-0.59494483,-0.3725219,-0.27995518,0.049034983,-0.49823403,-0.110797465,-0.53056246,-0.2704978,-0.6733415,-0.3675439,-0.17413245,-0.2250948,-0.24810278,-0.48820156,-0.44324404,-0.40427303,0.3504287,-0.3967617,-0.75321406,-0.43037298,-0.3607404,-0.67688006,0.2614906,-0.4650643,-0.37768254,-0.28748465,-0.65242183,-0.39631736,-0.26716718,-0.40593937,-0.3675729,0.7562946,-0.14993086,-0.376362,0.358394,-0.6108942])
net.state_dict()['block3.2.weight'][:] = torch.tensor([[1.59028471e-01,6.53460324e-01,-4.05589759e-01,-1.76099092e-01,-1.03983007e-01,-1.68273315e-01,2.53565669e-01,-9.94283240e-04,-2.28279512e-02,1.77716359e-01,5.76127395e-02,7.46126324e-02,-1.18568555e-01,-4.65330839e-01,-1.26770213e-01,5.44730276e-02,1.66465387e-01,-7.48861209e-02,-1.44991919e-01,-3.40566665e-01,-3.08277041e-01,-1.11320055e+00,1.77306026e-01,-3.18814307e-01,-1.54854685e-01,-3.96539181e-01,-5.46445072e-01,2.31569991e-01,-3.74594241e-01,-3.85123581e-01,5.48503697e-01,-1.64200351e-01,-1.93396583e-01,2.11544126e-01,3.23831886e-02,-2.76840806e-01,1.23611379e+00,-2.23871037e-01,-1.64608181e-01,6.86125576e-01,-1.08188111e-02,3.01567584e-01,3.88759762e-01,-1.59411967e-01,-3.08159202e-01,4.57184285e-01,-3.86862993e-01,9.02820900e-02,5.19859120e-02,1.21806160e-01,1.78636163e-01,3.59085314e-02,1.76037386e-01,-8.71888027e-02,-2.70159692e-01,4.86496687e-01,2.77543157e-01,3.63814563e-01,-5.17716855e-02,4.06318396e-01,8.59148800e-03,3.43772799e-01,1.95273206e-01,1.30626578e-02],[-1.0539917,0.6108781,0.21778478,-0.9614601,-1.1442943,-1.0264511,-0.8349162,-0.39350876,-0.52141744,-0.8308518,-0.79065573,-1.1143953,-0.9017614,-0.76513433,0.7242181,-0.68797344,-0.8735427,-0.8712117,-0.71403515,-0.80481505,0.02649493,-0.0784977,-0.74916613,-0.81231886,-0.96346664,-0.94271135,-0.3118158,-0.6609387,-1.1245468,-1.1006504,0.63706404,0.49914145,-0.6969485,-0.8228441,-0.9351228,-0.876773,0.6670077,-1.0825975,0.83986735,0.60211426,-0.89222205,0.499381,0.8353546,0.59593093,0.32406312,0.7514039,-0.9294996,-0.7604582,0.5283039,-0.8996901,-0.45373407,-0.681159,0.7966634,-0.5230616,-0.8203079,0.5332063,-0.73497796,0.4920026,0.60676855,-0.48860496,0.6154897,0.5998396,-0.4470341,-0.828714],[-0.09428768,-0.11777008,-0.02362071,-0.11125878,0.14653645,0.09556353,-0.37870634,1.0652188,0.09572344,0.19308655,0.588399,0.0290018,0.5071838,0.34811243,-0.464178,-0.10304019,-0.43085474,-0.01892772,-0.14287059,0.09978147,-0.811587,0.0138271,-0.03027128,0.06056832,-0.47508565,-0.5579045,0.02624474,-0.18031941,0.12115146,-0.04186479,-0.37559497,-0.41945785,0.17508693,0.55766475,-0.22016442,0.4071836,-0.2804395,-0.32879508,0.2658635,-0.4324882,0.36908293,0.43402514,-0.19420442,-0.21761145,-1.5518305,0.08578099,0.44973844,0.5055263,-0.09815734,-0.02229455,-0.25815797,-0.15105054,0.1282898,-0.3542678,-0.3817872,0.18555005,0.01881938,0.07338644,-0.00271732,-0.37029624,-0.02527199,-0.13571098,0.14331147,0.42873785],[-0.44309255,0.19053419,-0.06981142,-0.30652067,-0.63834006,-0.55026394,-0.6638558,-0.07980114,-0.36235908,-0.627745,-0.28066483,-0.29542285,-0.32671195,-0.4075041,-0.08061957,-0.3650267,-0.43065044,-0.27370176,-0.2465968,-0.3794609,-0.04160124,-0.09860998,-0.44966385,-0.1714135,-0.26807347,-0.3599361,0.1942642,-0.33029425,-0.36000988,-0.45717686,0.1552373,-0.6276456,-0.39831942,-0.18955348,-0.31263685,-0.15483052,0.8183723,-0.61225384,0.3370956,-0.5248658,-0.5319561,0.12976143,0.30609185,-0.21188262,-1.3004934,-0.6986979,-0.44736105,-0.27005243,-0.9288303,-0.62041646,-0.5817459,-0.44205725,-0.12216543,-0.28410363,-0.44062257,0.23055269,-0.44904268,0.3066524,0.18956468,-0.09165489,0.0995414,0.13373049,-0.5867764,-0.33975592],[-0.72080755,0.5883315,-0.16653262,-0.84470826,-0.85395473,-1.0810814,-0.39151388,-0.22893323,-0.52665424,-0.65094805,-0.51744133,-0.552625,-0.6322979,-0.44922924,0.46580347,-0.5716521,-0.60744655,-0.48811927,-0.346445,-0.77233064,0.11439747,-0.3970016,-0.63658625,-0.45840862,-0.5249823,-0.6295925,0.28628954,-0.4318666,-0.9733216,-0.80528134,0.51292557,0.33444607,-0.41893166,-0.40611702,-0.72274333,-0.5324432,0.5645732,-0.69389707,0.79774725,0.15036891,-0.6837867,0.38231993,0.34203997,0.40843466,-2.1566792,0.3065042,-0.5211107,-0.66508114,0.5947471,-0.7311364,-0.44720194,-0.48710093,0.53715235,-0.28677854,-0.6269006,0.25929195,-0.4828726,0.5585804,0.5269435,-1.2795746,0.4600796,0.488071,-0.7586884,-0.5881054],[-0.51921225,0.74376893,-1.0088272,-0.53457576,-0.99917495,-0.8156904,-0.49066085,-0.07872869,-0.49694934,-0.85408497,-0.5516934,-0.9058479,-0.50951785,-0.33155388,0.7080915,-0.41044083,-0.57307667,-0.534444,-0.5040028,-0.85909796,0.00487502,-2.3342123,-0.44510233,-0.5051276,-0.67477673,-0.4735325,-0.5136584,-0.2877631,-0.6938069,-0.666447,0.564135,0.37192336,-0.4535299,-0.4863562,-0.26246873,-0.6476116,0.5202496,-0.70170796,0.5589732,0.6184765,-0.76991457,0.5038204,0.70633715,0.49366865,-0.09883016,0.1545215,-0.5110655,-0.7517164,0.2724168,-0.62577695,-0.42310986,-0.50733984,0.38880977,-0.2972433,-0.40431336,0.4101967,-0.46405324,0.51958776,0.45003587,-0.29164866,0.3617805,0.40087467,-0.1245214,-0.51905406],[-8.31720829e-02,4.74242091e-01,1.93427622e-01,-1.55660525e-01,1.41717702e-01,1.39778763e-01,1.51545191e-02,-2.01286264e-02,-2.83025771e-01,-5.38365766e-02,1.04488708e-01,-4.84842695e-02,5.93725070e-02,-3.99432853e-02,1.81418106e-01,-1.44951329e-01,2.05360167e-02,-3.63159746e-01,-1.85875207e-01,-1.30980024e-02,-2.11061120e-01,-5.19864917e-01,-1.50751829e-01,2.41810940e-02,2.06675574e-01,-1.21000400e-02,-4.57000546e-02,-7.04814855e-04,-3.13021094e-01,5.38002364e-02,3.07387143e-01,-6.40152469e-02,8.72183740e-02,-9.92946420e-03,2.57045686e-01,7.66008720e-02,4.21967208e-01,9.81663913e-02,-1.47322088e-01,4.75562721e-01,-7.64547363e-02,-8.25616717e-02,9.63000432e-02,8.24187398e-02,-9.07287776e-01,6.61208481e-02,-2.40717009e-01,2.94145215e-02,2.53905803e-02,4.26920727e-02,3.24253142e-02,-3.40508521e-01,1.60259753e-02,-5.90386987e-02,-4.41053398e-02,4.29384798e-01,-6.52144402e-02,2.60194033e-01,-3.60714979e-02,3.16531628e-01,3.33057612e-01,1.75211877e-01,-6.75704360e-01,1.06036134e-01],[-0.09764937,-0.1066552,0.02071899,0.0539852,-0.26945126,0.07601443,-0.2478683,-0.05984673,-0.2402073,-0.03130561,-0.32512674,-0.18049435,-0.19818544,-0.11257461,0.0702044,0.19747975,0.04136507,0.02225751,-0.1029983,0.01634002,-0.27754873,0.21002173,-0.12626375,0.0235412,-0.1015186,-0.08658014,-0.57667273,-0.30459544,-0.04042022,0.01955476,0.15721428,0.08730382,-0.03159527,0.09201717,0.17558736,-0.37467635,0.5742596,0.11102593,0.02071615,-0.18921648,-0.05759265,0.05632327,-0.09926586,0.09864266,-0.69830096,-0.11531568,-0.28508008,-0.03379797,-0.03237246,-0.17467645,0.27729917,-0.05542922,0.24115448,0.00080798,-0.17952159,0.31111452,-0.2721606,-0.06774469,0.2820813,-0.12645002,-0.01689627,0.05078007,-0.06507863,0.10222557],[-0.33491352,0.57231426,-0.8931626,-0.34789926,-0.68284744,-0.65778095,-0.6219125,-0.03475516,-0.252039,-0.3176353,-0.27528986,-0.48906156,-0.49134043,-0.3440311,-0.31401482,-0.2714338,-0.34590507,-0.46780032,-0.33482566,-0.07153777,0.11082907,-1.5453999,-0.4489796,-0.30546448,-0.08340555,-0.38040224,-0.1669179,-0.24557284,-0.656383,-0.32573,0.2625237,-0.4440622,-0.49995255,-0.57826936,-0.5941056,-0.4554478,0.6197166,-0.41323623,0.519216,-0.03570291,-0.46771735,0.17805934,0.12942657,0.3463755,-0.14268167,0.15998183,-0.46588165,-0.39121565,0.5527589,-0.51652354,-0.36376595,-0.3809534,-0.43182415,-0.21204992,-0.4032331,-0.16617194,-0.43966466,0.19447504,0.5953166,-0.3006903,0.1659953,0.30381274,0.00747313,-0.14026536],[-0.29242408,0.13517353,-0.03368415,-0.42119917,0.6679205,-0.0714263,-0.21459612,-0.2620507,-0.49143764,0.00137882,-0.1927977,-0.13769737,-0.48385507,-0.4226647,-0.27325562,0.12430187,-0.31831357,0.58827347,-0.15912613,0.01161351,-0.85552794,-0.8200314,-0.01356394,0.53696847,0.3034172,0.09229058,-0.27641153,0.3940078,0.3705369,-0.1010317,0.07919035,-0.32476845,-0.49086815,0.40693912,-0.01871285,0.03433353,-0.15038323,0.0277893,-0.24165146,-0.06316556,0.36117315,0.2844172,-0.29738674,-0.30736443,0.01573373,0.15053326,-0.12165883,-0.4632361,-0.22248867,0.03526584,0.16674082,-0.3082529,-0.12461769,-0.37292933,-0.22168578,-0.19868639,0.22936465,0.16923936,-1.008267,-0.29421723,0.09871166,-0.26807612,0.12943487,0.34046382],[-0.20169033,0.20162742,0.05634623,-0.33838603,-0.1442083,-0.21969396,-0.39374897,0.2880885,-0.3114791,0.3350309,-0.13574132,-0.21262012,-0.57504755,-0.07561675,0.50727344,0.4147412,-0.22134745,-0.39835635,-0.28302294,-0.25119093,-0.38832814,-0.20299417,0.36298013,-0.10250564,0.42498726,0.00404179,-0.63370585,-0.03953882,0.16106042,-0.02203314,0.17330386,0.38549277,-0.40762833,-0.14180565,-0.21756004,-0.13199094,0.4596253,-0.03345998,-0.24068126,0.05122222,-0.36960986,0.17682293,-0.16353592,-0.17926656,0.14724925,0.08597905,-0.04380426,-0.03046983,-0.45585325,-0.14909498,0.14812241,-0.12127905,-0.24249722,0.03997611,-0.14844051,-0.21447693,0.3745397,-0.01191971,-0.36778575,0.13374625,0.06004581,-0.12809123,-0.2663077,-0.08493494],[-0.7504966,0.5665273,-1.4491931,-0.89741355,-0.78153497,-0.7024364,-0.71758306,-0.1070241,-0.43569148,-0.87030125,-0.66159165,-0.65295124,-0.6224559,-0.42246073,0.7584434,-0.501955,-0.5507056,-0.5887225,-0.27247113,-0.81835115,0.14896469,-2.20702,-0.5220038,-0.36698255,-0.84486705,-0.5981686,-0.33166602,-0.45759726,-0.97499526,-0.97131616,0.6375762,0.41502732,-0.7244761,-0.5803173,-0.54193765,-0.68005437,0.39821848,-0.791818,0.5914734,0.13443007,-0.6949595,0.6107266,0.74620664,0.30279374,-0.09213199,0.85955745,-0.66235244,-0.5653044,0.6740019,-0.702555,-0.4556795,-0.4760565,0.5451807,-0.46688083,-0.87621176,0.52399004,-0.33785903,0.35604638,0.5516903,-0.9954566,0.48172078,0.3058089,-0.2924477,-0.6026301],[-1.0981752,0.65515286,-1.6679289,-0.80833644,-1.0959766,-1.0278181,-0.88162374,-0.2555942,-0.3314127,-0.93986917,-0.53411365,-1.3273705,-0.80511165,-0.6468123,0.4005398,-0.6206021,-1.0296736,-0.8231101,-0.6050764,-0.7686327,0.02786906,-2.952155,-0.6100559,-0.72858393,-0.96005446,-0.90418494,-0.40847588,-0.7570145,-1.0251788,-1.0133479,0.46963066,0.6008197,-0.7515989,-0.788843,-1.0212511,-0.6679864,0.6981446,-0.8862496,0.78162146,1.0499215,-0.9538216,0.5182693,0.57491976,0.58217305,-0.18707436,0.8739106,-0.95107704,-0.7644061,0.44108963,-0.81805354,-0.25530335,-0.52834654,0.75022465,-0.5795442,-0.8615708,0.61550677,-0.63510305,0.45374516,0.5043385,0.3572989,0.45974526,0.26250958,0.20844285,-0.7937622],[-0.13549827,-0.21326587,-0.02490575,-0.062254,0.09172352,-0.04775819,-0.05297786,0.06721786,0.08220952,-0.05342762,-0.25640133,-0.33283347,0.07589235,-0.2791209,-0.09746234,-0.11937425,-0.11579387,-0.02823771,-0.3031038,-0.08825546,-0.26483974,-0.19445159,-0.12817746,0.13083178,0.0727136,0.11717815,0.31243345,-0.15819272,-0.2550807,0.04535021,-0.04780345,-0.19066896,0.08788951,-0.13923752,0.15207644,0.04505296,0.72084904,0.19479126,0.4595185,0.77119935,0.17888957,0.2556096,0.3680503,0.04522465,-1.1207684,0.3526053,-0.11599732,0.18000798,0.11816142,-0.06699152,-0.15404499,-0.11803305,0.17183146,-0.13180934,0.1422236,0.19841582,-0.31347135,-0.06238064,-0.15106697,0.09745909,0.04272583,-0.2156169,-0.27928853,-0.17983955],[-0.00335999,0.2738946,-0.16028844,-0.02998376,-0.01905017,0.16898684,0.30240628,0.7057533,-0.3514947,0.4426962,0.47871238,0.35830727,-0.19739832,-0.42833728,0.20015107,0.36918238,0.44343364,-0.19064006,0.6087444,-0.287854,-0.41286573,-0.5677948,0.31856108,-0.32735413,-0.16496918,-0.23214372,-0.5776325,0.4545892,-0.04239807,0.16955993,-0.2912591,-0.4233319,-0.24692811,0.24156098,0.13307782,-0.13571577,-0.3905626,-0.23038697,-0.23958564,0.1858778,0.16022694,0.0683152,0.5332755,-0.23617019,0.09469207,0.32721272,-0.33619317,0.48677966,0.22093736,0.04044245,-0.3109732,-0.03434019,-0.13340221,-0.0620711,0.06791711,0.03417603,-0.09845171,0.00623113,0.24442695,0.01647396,-0.2863211,-0.12973727,-0.03439089,-0.21940152],[-0.09833302,0.03127884,-0.24478757,-0.15679072,-0.5429808,-0.33430073,0.0532392,0.20348425,-0.16986805,0.42449743,-0.02328431,-0.29840896,-0.5012284,-0.21125308,-0.7215986,0.03999846,-0.17034678,-0.47334743,-0.3478624,-0.36943364,-0.41784492,-0.56231296,-0.44805685,-0.01477945,0.03392388,-0.04126065,-0.46782753,-0.06284308,-0.30791345,-0.55716246,-0.27357033,0.35335076,-0.09148958,-0.25777018,-0.4211908,-0.40625015,-0.29465196,-0.24381316,0.61954,0.0069699,0.23950526,0.39228848,0.1510216,-0.04628835,0.00412656,-0.09297322,-0.35494956,0.20981883,-0.72053945,-0.04835481,-0.01887662,-0.2229587,0.22853012,-0.33332765,-0.49323162,-0.46726277,-0.45866075,-0.4478995,0.03406161,0.02169948,0.03880698,-0.3502075,0.18106064,-0.19508982],[-0.3448139,0.31230164,0.28645134,-0.3687209,0.02478057,-0.32223815,-0.20410748,0.25709972,0.10342631,-0.3952615,0.06821191,-0.4610468,-0.23999898,-0.45653996,-0.15472347,0.04278867,-0.00174732,-0.34844682,-0.17828704,-0.12727442,-0.33249044,0.0019578,0.17058557,0.02967381,0.21511427,-0.34576887,-0.3456421,-0.20271865,-0.5020018,-0.4007488,0.14599268,0.37205738,-0.1955405,0.11887763,0.05477658,-0.26142594,0.42785507,0.15124732,-0.10642441,0.45818812,-0.05161953,0.4105774,0.36456072,0.04559904,-0.8699804,-0.21846794,-0.61368716,-0.27129376,-0.18481538,-0.01951834,0.08856827,-0.16035642,-0.39945117,0.06955864,-0.19352117,-0.09680322,-0.01702951,0.19329156,0.36135554,-0.23405476,0.34657362,-0.13261804,-0.20320128,-0.03918704],[-0.52083695,-0.03452855,-0.3457569,-0.13599147,-0.30226478,0.57312965,0.885461,0.6698421,-0.47471005,0.23000446,0.8730944,0.05943902,0.8936798,-0.53939927,-0.45295426,0.5312007,0.17138098,-0.59716177,0.6752837,-0.6540566,-0.7321878,-0.169885,0.3573241,0.59995174,-0.24586563,0.23675933,0.33818558,-0.28013593,-0.38567433,-0.37826452,-0.351071,0.06993178,-0.7056723,-0.45515773,-0.33662352,-0.07149507,-0.6741218,-0.65533996,-0.521767,-0.6654195,0.00350578,0.18159686,0.06232997,-0.43463027,-1.3005493,-0.39917353,-0.7210412,-0.31240058,-0.03633372,-0.87870574,0.00601877,0.08731349,-0.42650905,0.3079824,-0.9052741,-0.25002864,0.8171765,-0.21926382,-0.21164928,-0.15841855,-0.12355,-0.50449705,0.18146461,-0.03182993],[-1.0968626,0.38878193,0.15028337,-1.0252948,-1.0510625,-1.2512196,-0.99097747,-0.3362553,-0.37924194,-1.0104032,-0.8852636,-1.360476,-1.1555686,-0.7523495,0.790731,-0.83929306,-1.2984914,-0.9710535,-0.8070547,-0.8615135,0.0802884,-0.00586251,-0.7934353,-0.7898577,-1.0641899,-0.92692584,-0.18053289,-0.7023866,-1.4023658,-1.2278506,0.5568832,0.63836277,-0.7952068,-0.81357485,-1.0860367,-0.77525914,0.17139705,-1.0058606,0.9078935,1.1590306,-1.1371524,0.5910561,0.7365171,0.64372337,-2.8139205,1.0524163,-1.1812133,-0.92524713,0.51240486,-0.9657546,-0.3693328,-0.5024112,0.91408366,-0.6858591,-0.9652989,0.44830537,-0.83747935,0.39208513,0.591166,0.38050112,0.49469736,0.49170375,-1.3748765,-0.8841067],[-0.02642076,0.27564603,-0.44902426,-0.2718273,0.19866008,-0.18028174,-0.08946741,0.13470055,-0.189762,0.03382741,0.14358224,-0.25513864,-0.27439332,-0.5561172,-0.08528919,-0.13475057,-0.23033631,-0.21949147,-0.40502924,-0.0480425,-0.6488169,-0.41307783,0.36266226,-0.05614297,-0.25496307,-0.12345959,0.6541526,0.11641431,-0.24018623,0.02802671,0.00539316,-0.43113717,-0.21314642,-0.1415525,-0.33406913,-0.07122643,0.2237382,0.05053522,-0.26312634,0.13307829,0.01902496,0.01569692,-0.1043306,-0.24826011,-0.40892634,-0.10369953,-0.0122123,0.0987178,-0.2906224,-0.13742132,-0.28422624,-0.69052434,-0.41244158,-0.15710226,-0.23664308,-0.35491157,-0.23940766,-0.09912188,-0.26779708,0.09353612,-0.1385045,-0.10302361,0.02473664,-0.34335423],[0.08605755,-0.13353571,-0.71876043,0.01550358,0.39989334,0.16892374,-0.4969143,1.2071215,0.22084473,0.04405897,0.4320819,0.22870786,0.66071665,0.44977558,-0.38910308,-0.156332,-0.5484706,0.05682692,-0.23539473,0.2258211,-0.8154133,-1.3195813,0.08758486,-0.03184135,-0.36356357,-0.4103591,-0.2571095,0.03508301,-0.00160576,0.0056457,-0.47527975,-0.18722178,0.18010803,0.45639127,-0.46021956,0.35882854,-0.05540216,-0.2311334,0.36283377,-0.2319808,0.18108833,0.38008308,-0.17227037,-0.03037776,-0.04994927,-0.27404967,0.44632334,0.23909332,-0.21354803,-0.04137594,-0.16468798,-0.1786303,0.11220981,-0.5255359,-0.54407936,0.10291572,0.0604209,0.04101085,-0.04977638,-0.1775753,0.06836978,-0.09458037,-0.07943776,0.43399563],[0.12984087,-0.11061341,0.0264474,-0.14919025,-0.3984875,-0.18391877,-0.23288558,-0.35089415,0.1775651,-0.19630487,0.1853699,-0.17867509,-0.14214246,-0.3450514,-0.19098829,0.22111768,-0.13979623,-0.14764367,-0.12436885,0.13798928,-0.29263365,-0.07443602,-0.0151689,-0.28301808,0.02427552,-0.09225021,-0.08195092,0.2558929,-0.41668907,0.21240312,0.05488207,0.23251575,-0.11885715,0.22822188,0.06167305,-0.03248857,0.5464418,-0.01556594,-0.07747474,0.39210027,-0.02398705,-0.04986798,0.11298447,0.80715,-1.1381686,0.68065125,-0.22872964,0.02040294,0.06886115,-0.05481254,0.12218744,-0.01447699,0.02176975,-0.02636261,-0.35708538,0.21064122,-0.30066642,-0.24896881,0.2221982,-0.00611808,0.09333132,0.00908857,-0.27066156,0.01831752],[-1.081485,0.7630978,-0.23118341,-1.0966705,-1.2066283,-1.2808566,-0.7824716,-0.42907655,-0.60867393,-0.85994,-0.85907936,-1.2822576,-0.99586314,-0.79115,0.85529697,-0.8214668,-1.0452588,-0.9600731,-0.70751244,-1.008943,0.087548,0.02557454,-0.8302136,-0.8167547,-1.0256315,-1.0141963,0.25900775,-0.758327,-1.4119726,-1.23681,0.73400897,0.6079063,-0.8513991,-0.79434323,-1.1039172,-0.99777573,0.69438404,-1.0842186,0.97923523,0.7613224,-1.06474,0.60281575,0.8502663,0.69849545,-0.43623474,0.8929081,-1.1009372,-0.8967717,0.62729716,-0.9888251,-0.5064193,-0.8061208,0.8815446,-0.59230113,-1.067678,0.6188168,-0.88499856,0.5185879,0.6228749,-0.53886396,0.6424032,0.57356375,0.43811116,-0.8889681],[-0.12056921,0.5588378,-0.6513789,-0.403728,-0.28898838,0.10553153,0.03342839,0.02975282,-0.091028,-0.04841454,-0.17188983,-0.3665214,-0.12068077,-0.14072062,0.00846049,0.07631533,0.12192937,0.20359923,-0.27310622,-0.31644315,0.02856623,-2.1147175,-0.06870872,-0.02054393,-0.27217853,-0.22258212,-0.5563226,0.24360849,-0.14116839,-0.25705785,0.32755882,0.15693934,0.06331022,0.1328108,-0.17376718,-0.30766025,1.0849018,-0.2619425,0.5405426,0.69705975,-0.19229457,0.01749413,0.11993946,-0.19544755,-0.5750065,0.35085344,-0.08539259,0.02319864,0.4152656,-0.13270403,0.07587045,-0.03799625,0.18229003,0.0521251,-0.1680014,0.2726314,-0.31370652,0.25402582,0.07393675,0.0902729,0.13798635,0.02046942,-0.01874277,-0.06629277],[-0.600356,0.700455,-0.04469352,-0.5249575,-0.8145634,-0.50709856,-0.468174,-0.03819098,-0.30240968,-0.3902561,-0.2957926,-0.4462372,-0.5547023,-0.18193282,-0.679991,-0.2818386,-0.46179655,-0.35283688,-0.43151182,-0.3294667,0.10693784,-0.15770899,-0.586737,-0.3248193,-0.17137477,-0.59591174,-0.00447149,-0.37725526,-0.62418956,-0.3301002,-0.24767177,-0.7182441,-0.3919584,-0.49092656,-0.5595359,-0.2683319,0.2642023,-0.55319947,-0.06385157,-0.33086085,-0.54463404,0.20662275,0.1604254,0.26426432,-1.7391967,-0.20669371,-0.3827784,-0.3073298,0.48110884,-0.21709567,-0.4070162,-0.55523634,-0.04532675,-0.37270227,-0.6156056,0.4169217,-0.40871543,0.23888089,0.39616296,-0.34340066,0.44489893,0.29343542,-0.51796687,-0.39663634],[-0.63232243,0.0136688,-1.0257423,-0.23406261,-0.11366908,0.35176703,0.67296875,0.74586105,-0.27928537,0.25470373,0.6084834,-0.13953474,0.77464753,-0.3377185,-0.2866256,0.4542249,0.14434019,-0.31226185,0.5046901,-0.5595619,-0.5891861,-0.7653176,0.24045132,0.47484148,-0.03251733,0.01898836,-0.8133578,-0.50923526,-0.46976316,-0.41961524,-0.43755224,0.11534941,-0.640095,-0.60387415,-0.06310157,0.01492494,-0.4908488,-0.5322832,-0.2998369,-0.48382908,-0.23129585,-0.01969427,0.01851343,-0.40219828,0.23155154,-0.36402336,-0.8243896,-0.32886037,0.00589565,-0.64957136,0.14150825,0.13325301,-0.18530105,0.31620094,-0.6207952,-0.2747318,0.56407714,-0.11748228,-0.1775814,-0.09888235,0.05568092,-0.35916752,-0.4182473,-0.02527023],[-0.08907968,-0.07058658,-0.22156245,-0.09361688,-0.24377958,-0.3618676,0.11673988,-0.23901613,0.14232709,-0.29154226,-0.16727465,-0.16049339,-0.1632035,-0.20065148,0.69259024,0.3281995,-0.03483477,-0.02869505,-0.1308486,-0.5515465,-0.41230822,-1.1343416,0.1716143,-0.08003443,-0.21174869,0.07872804,-0.43590704,0.19130395,-0.28345394,-0.07020853,-0.4022245,0.26118636,-0.15830007,-0.05726159,0.26649937,-0.2027324,0.11197553,-0.02234627,0.48283607,0.677608,-0.22827908,0.3533332,0.11764283,0.55900925,-0.3156639,0.7999105,-0.17771375,-0.11430755,0.09767462,-0.06772128,-0.27364576,0.04706076,-0.14857672,0.02986598,0.00888932,0.41135234,-0.04296056,-0.07225777,0.15638992,-0.13256373,0.07806104,-0.10283262,0.27258673,0.22875504],[-2.31036916e-01,1.56969987e-02,9.68866423e-02,-4.96264189e-01,1.13342598e-01,2.74720043e-01,-4.20046896e-02,2.60534108e-01,-4.42615271e-01,3.41358602e-01,-6.84827030e-01,-1.21980093e-01,-1.31335929e-01,9.66259837e-02,-4.27575767e-01,6.35459900e-01,-2.80616611e-01,2.29723856e-01,-6.13085553e-02,-2.28966296e-01,-6.41642690e-01,-8.12714547e-02,5.29217184e-01,-1.51749581e-01,-3.22613046e-02,-4.45596874e-03,1.16145268e-01,4.92837548e-01,-4.89363909e-01,-3.25273387e-02,6.66706115e-02,-3.39442641e-01,-8.32760241e-03,3.49120140e-01,1.14234537e-01,3.70647699e-01,-1.65890083e-01,1.24255167e-02,2.29756191e-01,-2.10783005e-01,3.65411103e-01,-6.06856716e-04,-1.70706958e-01,-4.01537061e-01,-1.16765189e+00,8.00088570e-02,-7.76030242e-01,4.96769190e-01,-3.95704240e-01,-1.19279735e-01,-7.56338596e-01,9.97219235e-02,-2.82442328e-02,-8.48106503e-01,3.03236663e-01,-2.25299492e-01,4.19585817e-02,2.37562343e-01,-5.10496199e-01,-3.29722106e-01,3.78429979e-01,-4.77932356e-02,1.54931903e-01,4.70026046e-01],[-0.318414,0.08204682,-0.76513994,-0.67053485,0.12647589,0.3448467,-0.20526393,0.32866263,-0.5698011,0.38970077,-0.5000685,0.00741682,-0.0489274,0.24026172,-0.08318827,0.58663803,-0.27323374,-0.0123785,-0.12777983,-0.34373093,-0.8330692,-1.2689289,0.83362764,-0.35495722,-0.4816859,-0.07246737,-0.5376986,0.60135055,-0.6643064,-0.17494272,0.02721826,-0.2944332,0.20606032,0.17086604,0.2906025,0.39502433,-0.0859075,-0.22061144,0.12142655,-0.1759025,0.5262504,0.31320664,-0.15376776,-0.62958336,-0.04874539,0.24024545,-0.18622786,0.7047152,-0.47870305,0.09799188,-0.67117643,0.04878752,-0.11433072,-0.88875616,0.46695918,-0.38120058,0.06907415,0.17448308,-0.25756362,0.02234848,0.22271805,-0.01563457,-0.09899519,0.6766526],[-0.29750448,-0.20452394,-0.07123309,-0.12471499,-0.8013589,-0.64799035,-0.30245033,0.14473455,-0.09359322,-0.40288115,-0.78485173,-0.33561027,-0.62015337,-0.0711261,-0.5593298,-0.10211429,-0.5088531,-0.3068125,-0.20648636,-0.37570095,-0.05364936,-0.05314911,-0.4290238,-0.6268891,-0.34770697,-0.2326803,-0.03003291,-0.57588524,-0.3099167,-0.3327037,-0.1309169,0.26050875,-0.28077233,-0.37630635,-0.61736876,-0.27436578,0.6331427,-0.4659853,0.18861945,-0.30094162,-0.17491494,-0.38743716,0.00784346,-0.12249459,-0.90698236,0.3219512,-0.9251038,-0.12935442,-0.40350094,-0.4265852,0.18738264,-0.6398385,-0.03085131,-0.21414399,-0.4136465,-0.12483001,-0.75048286,-0.4047749,-0.00904423,-0.26588857,-0.05186586,-0.14641668,-0.3423066,-0.40133786],[-0.20146799,-0.27104747,-0.38342878,-0.1365638,0.02649402,0.23152329,0.48843995,-0.05989239,-0.06862067,-0.12016374,0.44089368,0.09911507,0.09227233,-0.40406573,-0.36206582,-0.2233025,-0.19243373,-0.22026895,-0.4840437,-0.59376025,-0.5173339,-0.63535506,0.12326471,-0.20300674,-0.64299554,0.07445941,-0.22430262,-0.35662785,-0.17866527,-0.12809193,-0.14838907,-0.4653868,-0.02510876,0.33582053,-0.4654241,-0.03115756,-0.25835007,-0.38237834,-0.04466144,0.0478134,-0.10633184,0.09115457,-0.27538612,-0.565053,0.24940363,-0.49410424,0.15776068,-0.02832373,-0.09495953,-0.23423523,-0.4256448,0.08047952,-0.34907037,0.17346835,-0.17433272,-0.48931077,0.4205617,-0.05236556,-0.37845957,-0.34119463,-0.38221636,-0.28439865,-0.21505149,-0.09532923],[-0.3820541,0.52610123,-0.60683453,-0.36852142,-0.79202896,-0.72423965,-0.23453426,-0.02906777,-0.5563395,-0.5175528,-0.27296287,-0.41889554,-0.15228966,-0.25008494,0.361202,-0.30524743,-0.45422775,-0.61833024,-0.20145835,-0.7644306,0.1818787,-1.3088126,-0.5440835,-0.48007798,-0.602632,-0.39678434,-0.22539033,-0.8500965,-0.5236121,-0.7370988,-0.20305131,0.30110055,-0.1994314,-0.31961194,-0.48429576,-0.42041385,0.12763056,-0.61332434,0.4563952,0.11347314,-0.61391836,0.40171766,0.04878332,-0.8873222,-0.09754162,0.00607656,-0.73501754,-0.3254745,-0.60152584,-0.3933573,-0.250662,-0.5279488,-0.08818001,-0.3584917,-0.65676427,0.26990238,-0.5101723,0.15632285,-0.46554098,-0.16485706,-0.33508274,0.3561237,0.12000646,-0.60062265]])
net.state_dict()['block3.2.bias'][:] = torch.tensor([-0.6437253,-0.14790297,-0.59597266,0.68462634,0.4447321,1.0146353,-0.72539556,-0.22065307,0.6010244,-0.56903934,-1.1182951,0.554421,1.7591577,-0.956438,-0.48821747,0.12078097,-0.1590207,-0.28027573,1.492066,-0.14424063,-0.4981463,-0.6470756,-0.06525908,-0.6201724,0.78962106,-0.625907,-0.22079282,-0.5172899,-0.7659096,1.0534472,-0.11185222,0.3298632])
net.state_dict()['block3.4.weight'][:] = torch.tensor([[0.04296314,0.29490873,0.34601757,-0.07801319,-0.0900257,0.03614852,-0.15311776,-0.04625572,0.09097442,-0.16631648,-0.1004113,0.08630995,0.10721777,-0.19420695,-0.05264195,0.05469554,-0.10851382,0.29912972,-0.10920961,0.44873053,-0.29563063,-0.20763955,-0.29201567,0.2461575,-0.06000502,-0.285076,0.16595313,0.33025733,-0.27742347,-0.07273751,-0.22391534,0.06965009]])
net.state_dict()['block3.4.bias'][:] = torch.tensor([-0.006232739])

def test_agent_2(observation,configuration):
    start_time = time.time()
    net.eval()
    with torch.no_grad():
        pol = alpha_MCTS(net,observation['board'],configuration,start_time=start_time,root=True)
    action = int(np.argmax(np.array(pol)))
    if observation['board'][action] != 0:
        action = random.choice([c for c in range(configuration.columns) if observation['board'][c] == 0])
    return action

In [ ]:
#@title ***2-step Lookahead*** - test_agent_3
from numpy import reshape, argmax, add
from random import choice

def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return new_board

def find_scores(board, scores, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  # print(moves)
  for index in possibile_moves:
    # print(index)
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores

def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        return index[1]
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        return index[1]
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        return index[1]
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        return index[1]
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        return index[1]
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        return index[1]
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        return index[1]
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        return index[1]
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        return index[1]
  
  return -1


scores_player = [2,2,2,3,3,3,4,4,4,6,6] # 11 possibilities
scores_opponent = [-2,-2,-2,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    
def test_agent_3(observation, configuration):
    # print(observation.board)
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    # print("\nStart of Turn " + str(turn))
    scores = list(0 for j in range(7))
    player = observation.mark
    opponent = player % 2 + 1
    # print("Current Board:")
    # print(board)
    moves = possible_moves(board[:])
    # print("All Moves = " + str(moves))
    
    # Only Move
    if len(moves) == 1:
      # print("Only move")
      return moves[0][1]

    # Offensive and Defensive
    our_choice = win_check(board[:], player)
    if our_choice > -1:
      # print("Seizing at " + str(our_choice))
      return our_choice
    else:
      our_choice = win_check(board[:], opponent)
      if our_choice > -1:
        # print("Blocking at " + str(our_choice))
        return our_choice
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      if len(moves) > 1 and win_check(next_state(board, i, player)[:], opponent) > -1:
        # print("Removed " + str(i))
        moves.remove(i)
    # print(moves)
    # Last resort
    if len(moves) == 1:
      # print("Last Resort")
      return moves[0][1]
    
    # board = reshape(observation.board,(6,7))
    # print("Current Board: ")
    # print(board)
    # Scoring Moves : 2-step Look ahead
    value_opponent = list(list(-10 for i in range(7)) for i in range(7))
    # print("All moves = " + str(moves))
    for index in moves:
      # print("Current Index = " + str(index))
      board = reshape(observation.board,(6,7))
      next_board = next_state(board, index, player)
      # print(next_board)
      if win_check(next_board[:], opponent) > -1:
        value_opponent[index[1]] = list(-10 for i in range(7))
      else:
        next_moves = possible_moves(next_board[:])
        # print("Next Moves = " + str(next_moves))
        value_opponent[index[1]] = find_scores(next_board[:], list(0 for j in range(7)), opponent, scores_opponent)
      # best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    # print(value_opponent)
    final_value = list(sum(value_opponent[i]) for i in range(7))
    best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
    # print("Final Values = " + str(final_value))
    # print("Best Moves (no Priority) = " + str(best_moves))
    if len(best_moves) > 1:
      priority = [0,1,2,3,2,1,0]
      # final_value = list(sum(value_opponent[i]) for i in range(7)) + priority
      final_value = add(final_value, priority)
      best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
      # print("Best Moves (with Priority) = " + str(best_moves))
    our_choice = choice(best_moves)
    # print("Choice = " + str(our_choice))
    # print("End of Turn " + str(turn) + "\n")
    # our_choice = choice([c for c in range(configuration.columns) if observation.board[c] == 0])
    return our_choice


In [ ]:
#@title ***3-step Look Ahead*** - test_agent_4
from numpy import reshape, argmax, add, copy
from random import choice

def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return new_board

def find_scores(board, scores, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  # print(moves)
  for index in possibile_moves:
    # print(index)
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores

def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        return index[1]
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        return index[1]
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        return index[1]
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        return index[1]
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        return index[1]
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        return index[1]
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        return index[1]
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        return index[1]
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        return index[1]
  
  return -1

def scorer(board, player, opponent, turn, scores_player, scores_opponent):
    # print(observation.board)
    # print("\nStart of Turn " + str(turn))
    scores = list(0 for j in range(7))
    # print("Current Board:")
    # print(board)
    moves = possible_moves(copy(board))
    # print("All Moves = " + str(moves))
    
    # Only Move
    if len(moves) == 1:
      # print("Only move")
      return moves[0][1], "only"

    # Offensive and Defensive
    our_choice = win_check(copy(board), player)
    if our_choice > -1:
      # print("Seizing at " + str(our_choice))
      return our_choice, "seize"
    else:
      our_choice = win_check(copy(board), opponent)
      if our_choice > -1:
        # print("Blocking at " + str(our_choice))
        return our_choice, "block"
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      if len(moves) > 1 and win_check(copy(next_state(copy(board), i, player)), opponent) > -1:
        # print("Removed " + str(i))
        moves.remove(i)
    # print(moves)
    # Last resort
    if len(moves) == 1:
      # print("Last Resort")
      return moves[0][1], "last"
    
    # board = reshape(observation.board,(6,7))
    # print("Current Board: ")
    # print(board)
    # Scoring Moves : 2-step Look ahead
    value_opponent = list(list(-10 for i in range(7)) for i in range(7))
    # print("All moves = " + str(moves))
    for index in moves:
      # print("Current Index = " + str(index))
      next_board = copy(next_state(copy(board), index, player))
      # print(next_board)
      if win_check(copy(next_board), opponent) > -1:
        value_opponent[index[1]] = list(-10 for i in range(7))
      else:
        next_moves = possible_moves(copy(next_board))
        # print("Next Moves = " + str(next_moves))
        value_opponent[index[1]] = find_scores(copy(next_board), list(0 for j in range(7)), opponent, scores_opponent)
      # best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    # print(value_opponent)
    final_value = list(sum(value_opponent[i]) for i in range(7))
    best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
    # print("Final Values = " + str(final_value))
    # print("Best Moves (no Priority) = " + str(best_moves))
    return best_moves, "steps"

def test_agent_4(observation, configuration):
    scores_player = [2,2,2,3,3,3,4,4,4,6,6] # 11 possibilities
    scores_opponent = [-2,-2,-2,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    player = observation.mark
    opponent = player % 2 + 1
    best_moves, category = scorer(copy(board), player, opponent, turn, scores_player, scores_opponent)
    # print(best_moves)
    # if str(type(best_moves))[8:-2] == 'int':
    if category != "steps":
      return best_moves
    else:  
      our_choice = choice(best_moves)
      scores = [0 for i in range(7)]
      for i in range(7):
        if i in best_moves:
          for x in range(6):
            if board[x][i] == 0:
              move = (x,i)
          next_best_moves, category = scorer(copy(next_state(copy(board), move, opponent)), opponent, player, turn + 1, scores_opponent, scores_player)
          if category == "seize":
            scores[i] += 10
          elif category == "block":
            scores[i] -= 5
          elif category == "steps":
            scores[i] += 1
        else:
          scores[i] = -10
    best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    if len(best_moves) == 0:
      return best_moves[0]
    else:
      scores = add(scores, [0,1,2,3,2,1,0])
      best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
      return best_moves[0]



In [8]:
#@title ***2-step Lookahead CNN*** - test_agent_5
from numpy import reshape, argsort, array, dot, e, shape, argmax, add, copy
from random import choice

wih = [[3.6920327192794615, -7.244057919842115, 4.233188964719989, 1.8043895415697038, 4.58275418359571, 3.0131849314057613, -0.16333164174715767, 1.503592951719824, -0.3740112461577557, 0.6063470379383881, -0.34058090813975045, 0.7268186530088414, 1.5658032501071046, 0.18987026349696098, 0.8046410738404773, -2.1380941332634924, 0.4458324185420854, 0.2537056973463159, -0.37465373092949333, 0.2581530173361692, 1.5680692392766982, -0.4100501138774406, -1.2876112663665622, 0.3239536985421265, 0.7780142197046687, -0.7719841932889516, -0.9447582119185367, 0.8995173952001375, -2.5284837441189607, 3.263793927397769, -0.757714515981968, -2.1050619497503136, -1.934786996437022, 0.4019735245550619, -1.173217057140023, -3.4384540793731104, -1.995033685709947, -3.8777044508161347, -2.852919811855892, -3.828481323615157, -3.3978379286326827, -2.668429701524594],
[0.26252528770641265, 0.08537400836603515, -0.10583574069138049, 0.26011879115402947, -0.16486155895725668, -0.32940383007750407, -0.39519567370767267, 0.012206109224897545, 0.18182075606524437, -0.07279158019367832, 0.07328013001363921, -0.020362456654926697, -0.889062271933661, -0.12553931425021, -0.15094403181432214, 0.09463315970201755, -0.6180914196503561, 1.2973410578338371, 0.2634412634772544, -0.12734773977264705, -0.2432061585832564, -0.009503360153626503, -0.36897325660628777, -0.5772582206660327, 0.733478272818461, 0.918168214528241, -0.5959710404484699, -0.35578815471694364, -0.6462851028879573, 1.069803686906628, -1.193053500911227, 1.0538317422891812, 0.41675342642891283, 0.7987953063510606, -0.3967371364903907, -0.46069080177891564, 0.36976799663146753, -1.7898386478968409, 0.1941470776331503, -0.5243041669212902, 0.6889454127318864, -0.8405579260938107],
[0.17752335014114676, 0.17974957996128155, -0.24931626340358432, -0.025959167377366622, -0.051093346564727686, 0.11601250092831167, 0.1729329314280967, -0.1490529131384966, 0.010170475089422546, -0.4544952056244338, -0.12196302108374676, -0.35512680137277636, -0.252463015066149, -0.059282670561936465, -0.5593973928240582, -0.38538941081096484, -0.7810787006207537, -0.2669608572773698, -0.5599957604657106, -0.06556342596819804, -0.3154079032746093, -0.002365838187085241, -0.26871324187568907, -0.9302188240611856, -0.6281326149823561, -0.6454672599450898, -0.4641170537249279, -0.25781368212188854, -0.5206421421497552, -1.0145484985207556, -0.6844176553770029, -0.08592805942340521, -0.6094882877835573, -0.7245178858541936, -0.8829521215674083, -2.463312468966375, -1.8274421722587624, -1.9978042615841989, -2.400400688071654, -2.2677830503363667, -1.6197895116289702, -2.0418136055728837],
[0.17925667061942868, -0.01574420965574411, 0.06828259303294873, -0.2183115233295555, -0.15204684786836226, -0.024672366174690003, -0.23757204877973492, 0.1503300689499923, -0.3496383952777234, 0.09487247903016591, -0.100380375791682, -0.14218674789283706, 0.024159892576882898, -0.2901808258295947, 0.37597826390366457, -0.2481520187971826, -0.14628782343165112, -0.2184532151376845, -0.2387585238917683, -0.1792384366229478, 0.011970137604445277, -0.09338497844811665, -0.15587011029095715, -0.5552241895385753, -0.33999018975924855, -0.401410874861212, -0.19631039734074476, -0.2262417656931879, -0.5839808202631326, -0.4786145838917975, -0.7172293722185812, -0.011984660661584915, -0.4263857610641744, -0.5845990734661946, -0.7835368296633993, -2.152675459189594, -1.8579732548073975, -2.060097328605771, -2.4045872548962333, -1.7443484987376132, -1.8116698856974087, -1.7906034252160008],
[-0.1981038730883346, -0.18276408685198015, -0.101808368204481, -0.12814079829956299, 0.10282372801747129, -0.044741337106355285, 0.040513965596121984, -0.19212601158809034, 0.06263110153940937, -0.42909670588867654, 0.12229452008009878, 0.09525428146128437, -0.12685268179097578, -0.08130523576379052, -0.01824211403821459, -0.48274825604280863, -0.24091002654990296, -0.06859233679534774, -0.04382087351652544, -0.27022140902796327, -0.04632327360369071, -0.33374006867085015, 0.31722552422364175, 0.13268107058694706, -0.3720490998354789, -0.26696889174336713, -0.11731463822251012, -0.1445592899112771, -0.7564575790864035, 0.3738989652374938, -0.582733072318517, -0.6092280482428568, -0.43695680879132043, -0.6671391214801992, -0.6458436481859038, -1.3018984186872413, -0.6513257206555594, -2.213320042175003, -1.986012210254286, -2.1467673062611725, -1.4021256590108693, -1.9634111721606389],
[-0.008314895052833936, -0.1787864717648438, -0.004949101981369209, -0.07521845142297474, 0.1468092884774125, -0.05333205274831611, -0.08620343780492468, 0.01863809495472172, -0.2619596483469773, -0.32279764236116826, -0.39187485274770417, -0.024864117564070125, -0.2176997384164171, -0.2177538476483432, -0.03411048584896557, -0.42164237171789737, 0.1646653717230308, -0.3013835283676457, -0.22511994858407292, -0.39921890944782734, -0.1642482569353005, -0.2762367052926482, -0.3194013997143978, -0.34413314616230195, -0.44772617931140896, -0.25483752920537134, -0.3657057031888192, -0.20722087976951023, -0.4752167471932325, -0.3614797647049482, -0.41526314686527205, -0.22939055539084685, -0.47702117402014227, -0.5891719769332915, -0.6145286856470878, -1.7903368757171108, -1.3227052175203946, -1.5910696869092136, -1.9825816132484555, -1.7005408635317198, -1.5647571816103063, -1.6827372107923628],
[2.5362053056966802, 2.4058538466356363, -1.8181980115978533, -1.0157103963511027, 1.5936430485040112, -3.936557008038382, 1.8024356045166516, -0.7373541241903451, 1.980090043395433, 3.498851332743481, 1.3691659531887401, -0.10865582215602089, -3.418876272738125, 0.5446006594556551, 0.4521303692465155, -0.03338818031827306, 1.0080689692022002, -4.727554549069765, -0.5156566571338314, -2.1572721003562134, 1.4420987262814025, 0.9748571160859978, -3.05372985110104, -4.718452359605314, 0.19316299912331644, -1.702190212913726, 1.123337919304037, 0.22954277542855542, -0.4421971235612093, -2.167123844818533, 0.576713787704928, 0.8941009797717341, -3.81236792407585, 1.7330114176193652, 0.5619854804220318, -3.257597570831327, -2.6120988942491885, -1.6448220564376639, 0.6111504170383164, -8.267751871894802, 1.7753651210442558, -1.6073162220773556],
[0.10629962930100126, -0.1444316769421932, 0.015433074992264635, -0.19649451237422771, -0.038733065348265094, -0.1025840501784094, -0.14395070860067505, -0.33595686344777187, -0.11988828777952147, -0.006998484423895029, -0.3280407711450199, -0.09832493694391696, -0.21498088269613552, -0.15481907942274448, -0.3352401180475094, -0.08631333038265747, 0.3090922953670953, -0.6248769770569869, -0.2648724027533989, -0.6475531962633709, -0.32721885169776926, -0.7081243445251159, -0.1750386775792655, -0.2509338344376779, -0.08314461299210942, -0.34583589823076855, -0.2869768172368614, -0.07406201117762692, -0.5981335488832001, -0.5262505215165822, -0.09352948155850556, 0.3502001598696306, -0.5003216674595267, -0.7005547507303147, -0.43260704504695674, -1.3843603552984234, -0.9085833121975441, -0.9026092935343676, -2.069755274857403, -1.2678624966624832, -0.741924325748288, -1.001383606648032],
[0.20255095607572876, 0.09610014667650126, -0.13941477010009215, 0.06394629371160314, -0.11357773634769032, -0.15686829111494088, -0.14276753407110504, 0.14420290944312766, -0.037956468615229125, -0.05915506090059916, -0.24433405301838312, -0.09762655818333123, -0.11970848930679874, -0.09270336364035013, -0.05193078290897494, -0.34080132582382955, -0.16379329673909937, -0.29890473923375405, -0.07226437126656088, 0.01720309805878413, -0.12270505720935286, -0.2869931523411432, -0.19017300848645918, -0.22894278675309745, -0.31421105590953585, -0.43581393219712355, -0.34307887882781324, -0.2909387677143875, -0.7199026488963939, -0.5840416374101399, -0.37653834856992846, -0.4664741500404451, -0.2759214712924747, -0.37690363597558374, -0.5173357374085191, -2.1972930859247044, -1.7168601222289785, -1.8544342331963277, -2.361383691620973, -1.9078100896061192, -1.7151042531785987, -1.7640395115521332],
[0.16792873408204545, -0.06038443187205756, -0.338743958478707, 0.09682311091252631, -0.09912042096047202, -0.04271029010441317, 0.14728147216054902, 0.09581963684632398, -0.09842098776509466, -0.3083211244736272, -0.11543249129359406, -0.23657850251573573, -0.22345331932141976, 0.05236406503712367, -0.03793592924029281, -0.22810600539249423, -0.2525186799762783, -0.00565562365294716, -0.2911155452407524, -0.16902613040168293, -0.13505478142525135, -0.3581840706471435, -0.4215346001717991, -0.21486887580441316, -0.15794442826370442, -0.4049605029415886, -0.4645872902837102, -0.40353362485836014, -0.8059720195899872, -0.37799042838133284, -0.5320031069574915, -0.3728961493472302, -0.46960957005256115, -0.49501917290027136, -0.748000077250521, -2.615289011142299, -2.227891403374709, -2.619514076325971, -2.410939143652046, -2.6099943927613825, -2.5101007993783293, -2.5122513480140887],
[-0.09087546551475865, 0.05469744443480035, 0.12360202951739449, -0.18239335947383725, -0.11698378641567339, -0.004934753783868976, -0.10282407249710168, -0.2756392412854901, -0.15791156700856884, 0.15942375869255276, 0.058601374908208936, -0.24955503807497975, 0.06928370515826561, 0.06770423294257914, -0.08379989757888545, -0.17573311461380864, 0.09061406510463524, -0.20587226219210344, -0.011780579194252435, -0.12381337714579015, -0.07940250729607617, -0.1347910083959297, -0.24979394365684124, -0.1694182854456895, -0.16593746313980268, -0.37363872143035726, -0.35273437523558443, -0.2592432306422306, -0.6911707351606581, -0.5080072957974445, -0.486972971328111, -0.3922435449523617, -0.4725027810958101, -0.5299996892112904, -0.744856274411746, -2.4584310948515067, -2.1182584386161736, -2.4906094313928215, -2.738555289763694, -2.399103752238234, -2.2729993678137084, -2.4477905681851215],
[-0.06066769951479112, -0.1298750812173647, -0.24598169191018965, -0.1004097037734866, -0.0828247361362373, 0.05010280529763124, -0.11276752861333186, 0.046336950169430596, -0.3513899868744168, -0.4244017394796529, -0.012985182370380948, -0.060248235580467054, -0.2673682410754724, 0.058572703520683826, 0.04742473719868934, -0.23743655830538127, -0.5261690984275181, 0.5296103072280637, -0.2500337706436513, -0.21362677879784236, -0.22572841911923686, -0.4291359926752966, -1.0556583331469374, -0.5108879273694872, -0.03951805681872705, -0.46632825815438594, -0.8029218406170845, -0.24845764013502383, -0.07069327533195527, -1.0748051190370136, 0.6953451783092637, 0.23613223590354687, -0.39941417518918076, -0.5987686534495663, -0.6625687630555238, 0.35698015438843284, -0.19226042751765166, -0.7691126834077132, 0.9162034845677918, -1.8209754256299175, -0.8390794397512208, -1.9648395936663794],
[0.5055894990696509, -0.30700102589094025, -0.756403287453546, -0.46108111060852064, -0.768482674333231, -0.4319157409931807, 0.31417344707235323, 0.14714314130618372, -0.4901511865914631, -0.5883066839026314, -0.3881320300039488, -0.32475916001974797, -0.649317956914093, 0.32146885152546245, -0.26724739820728866, -0.8105442991363302, -0.32458539982348095, 0.15042976239768321, -0.060699331871550355, -0.6349176628094743, -0.14200927263136426, -0.5081750524698221, -0.8169995220258063, 0.2772471123372893, -0.023332239093072363, 0.3005263755465825, -1.139798737475903, -0.9929749468016954, -0.40451483630836527, -1.9600180319077003, -0.008693765513740605, -0.548576446229067, 1.6402680350371224, 0.14093131089406813, -0.5243594180516029, -1.9841581853368961, -1.3667499420791176, -0.11189112458368686, -1.434819540921151, -1.620170219834856, -0.4256793528846523, 2.481070934375057],
[0.08332091698088577, 0.0872268027014748, 0.12728588011944555, -0.003436077251087078, -0.04234222333007598, -0.10989101809393775, -0.04816431561231924, -0.35195099992295137, -0.02782549618992084, -0.049105212372008614, -0.09462746294121745, -0.07281555747243268, -0.0906549378856271, -0.08698630636150206, -0.10552682549797025, -0.2987359850220972, 0.013342621646104615, -0.34642563374093477, 0.0201023898026383, -0.07057568694415192, -0.33431192184141645, -0.38148061601911215, -0.18648681225310118, -0.19279436690186016, -0.672824183512421, -0.1194710099670246, -0.011493916463980203, -0.18145235780090113, -0.7812251156867673, -0.5403945733363654, -0.7572436016072082, 0.11628387380883305, -0.45326912127547125, -0.6227998733255093, -0.10223563918757411, -2.0177882968679586, -1.2061818754390334, -1.7546858373639092, -2.0544623656678906, -1.2055290692199174, -1.3094742127356958, -1.1443517494676776],
[-0.06546975549232886, -0.3907193681093094, 0.27207839061110695, -0.6879538570022286, 0.13565851331195056, 0.12491738123465047, 0.12815428199238577, -0.12245147628329284, -0.06706176994281718, -0.23248786060401505, -0.5061190990112547, 0.12650589826288453, -0.04526539459099698, -0.2542709093211703, -0.06809903914652814, -0.4526570217473938, 0.4147214289416033, -1.227106533677836, 0.5364105016338285, -0.571662415894246, -0.28729016271267954, -0.3645965025766386, -0.016953417592223564, -0.30278252993971366, -0.6170208120548321, -0.15184499081282188, -0.14244641512355644, -0.5263144461656848, -0.7764201874991881, -0.8124612144798187, -0.24607581385064375, 0.040450595813110966, -0.8579359923620178, -1.0406247970995313, -0.64411938408346, -2.8750633238408456, -2.402422553310893, -2.678357135173968, -2.7973381140235545, -2.6308405782975455, -2.340604699392349, -3.0984768075139204],
[-0.17345382458779637, 0.43452663398412283, -0.20703775228575713, -0.1737200662964857, -0.31228273761922615, 0.11109628587686933, 0.3454727531564346, 0.009835515394316341, 0.20489588047098303, -0.0064692317631280085, -0.2404051874532601, -0.359331921435166, -0.09172890900516716, 0.2365967283438156, -0.14371989880372715, -0.1658852440046037, -0.24313937056358728, -0.15863378466185094, -0.1006307006783972, -0.16842824850001445, 0.030465955126264505, -0.22353075962763164, 0.06988137384012487, -0.22639242551384903, -0.2037779109471093, -0.15771556368554637, -0.185146032235658, -0.34769563983283025, -0.24127187366053818, -0.29694471208181455, -0.08522613982607996, -0.30506225287160765, -0.16156239838430003, -0.5863162695926124, -0.4678016134409775, -0.31553991856320085, -0.7267620623169766, -1.0640065687309224, -1.397221818004542, -1.111524583347383, -0.9134042856143889, -1.0043944745502058],
[-1.143395701175637, -1.2061107124116015, -1.348396939877448, 2.4586851143050112, -1.4535431519949684, 0.12964971843445772, 2.9177204835833446, -0.7125432431642184, -3.61001502217337, -3.758178102675495, 1.4156624454764772, -2.4595456948545933, -1.5667351764971864, 0.484146870009338, -2.5582705144622366, -1.44825379854444, -1.8217580763029353, 4.806030929621572, -2.581881724078662, -0.6325121940873261, 1.589620057392546, 0.24656881271644698, -1.0714380014607243, -1.2550241872222472, 0.7478170619939987, -0.5618398355345191, -0.6593406778431484, -0.28930136248451477, -0.27953114512100674, -1.0968096461065457, -0.4262013285809544, 1.7694291290139703, -1.3425340770060676, -0.2481958092585689, -0.1384247171410796, -0.18962815466696392, -0.46839675558503835, -1.8348223419671412, 0.540257602558185, -1.051753791502515, -0.8756868417709948, -2.030109018856068],
[0.1955612625737871, -0.627335214799185, 0.06509641554235851, -0.13761491252278593, 0.2874811089593937, 0.007748028911928659, 0.2340252038135999, 0.030796332550254355, -0.29379093308240517, -0.4383505186384552, 0.13146894974680054, 0.22788864980970225, -0.04349479758651049, 0.09779460394030515, -0.06503219174848968, -0.39421121076624116, -0.3138488361462421, -0.14601431286188665, 0.585163327329306, -0.39424056469987445, 0.07624540448097343, 0.09166609452497171, -0.427818286801801, 0.2251761678850975, -0.41817788575203035, 0.22268103672170636, -0.15941113295516407, -0.23648135036800458, 0.7029239217941758, -0.602913896253851, -0.5189254223078476, -0.34407500910977007, -0.3592456108571896, -0.5750456281810952, -0.35351798603435275, -0.4130187838379706, -0.7136166748822395, -0.7772844677756322, -1.5508614556899587, -0.5379382487751556, -0.2471568284370135, -0.9718577434443137],
[0.08537190412027047, -0.6423703967299358, -1.1090571892413572, -0.32362512632946655, 0.3385214348356562, 0.6939336415150053, 1.0755461728437596, -0.023726122706929222, -0.4718159787320922, -1.2176880078748087, 0.2822189738761779, 0.2268741425836726, 0.3725982356184618, 1.1474727237307119, -0.26986363145280573, -0.18520598633210889, -0.5205597448445656, -0.7235831630242093, 0.0844246780175919, 0.28292647112583336, 0.3784691482311491, 0.28634285626801265, -0.1747884990864307, -1.0175410049286457, -0.1645818059894885, -0.8473030001146239, -0.5736156453980109, -0.12958266023535225, 0.4885825510752945, -0.8164930692609704, 0.0926691459088164, 0.35585505529718486, -1.2922177608009833, -1.00807551439433, -0.7420278944732019, -1.5362948042765205, -1.502605848756868, -1.5813000658641438, -1.4002071829733447, -2.6922450768703863, -1.8004008415580681, -2.581058172521485],
[0.44771867718504027, -0.3714198202648779, 0.46911486962924326, 0.3794118183098849, 0.3545569412389165, -0.0536025191484126, -0.06546881803716177, -0.0128424228508469, -0.29585764383546537, 0.49694956706539656, -0.11384439911968758, -0.03397497969722869, 0.9560005597921628, 0.26854512957635623, 0.364621435722468, 0.3823365124975977, -0.22477798618118341, 0.1580634238798518, -0.39917775043960024, 0.1770609212556175, 0.2696061257769195, -0.10489933591032814, -0.6829499325091896, 0.07305194040235707, -0.4124723437552714, -0.589857672024159, 0.2839524347718052, 0.0868781107877945, -1.2023938417751234, 0.9280017433719672, -0.010405635328648799, -0.5555348593079876, -0.8157263603377649, 0.21338746369164913, 0.29609775226193924, -0.5518317553359047, 0.6846176976723256, -1.0179603788455545, -0.7177073059826781, -1.8748698588137913, -0.3426283364443814, -0.5483981032499297],
[-0.015371088175939055, -0.12465011771785577, 0.04518214814753919, -0.05043108441036863, -0.1928277782200663, -0.055716902119186396, 0.07265622743044356, -0.16523280431223747, -0.0806570044697366, -0.10409452185243415, -0.25756021552452746, -0.21201827388747863, -0.034367084639593454, -0.07548061577034165, 0.08205615032027552, -0.01777489109343277, -0.13586946726371982, -0.27262993699940785, -0.12142553987971033, -0.354523857647453, -0.4096550919171272, -0.3602836897081539, 0.009076017378329238, -0.2834459108323818, -0.524514268794333, -0.2857012360026934, -0.3851566028248068, -0.0928745446293783, -0.7573602484066627, -0.5282376138718411, -0.6032470361831146, -0.4691225665913047, -0.44923757771755873, -0.33292894419484936, -0.535930471396437, -1.5498866644593308, -1.2484236098833208, -1.7149282007692512, -1.702089466197689, -1.5277986856267272, -1.0687324770407154, -1.802178882372064],
[0.024865342073357333, 0.02054422326917201, 0.09695076691084502, 0.21557118074299364, -0.044670640962989776, -0.1389714288281299, 0.10238334005837002, -0.06611702394405143, -0.12854947921522825, -0.17928148990511092, -0.21853198692784168, -0.12452340768442001, -0.12536235179791416, 0.0876308908154015, -0.0388218463742762, -0.15253733052620624, -0.36499141250508604, -0.0900164324107219, -0.11181282094640689, -0.3135793246327136, -0.07041062535960804, -0.241415705323314, -0.3431018249129747, -0.3429072785412178, -0.5243956893558539, -0.3623773757924299, -0.43088987782414206, -0.2180727502427593, -0.3273694829913016, -0.5842092789260798, -0.34297286182759346, -0.3520949136782254, -0.4049144415236024, -0.5632498777153272, -0.6879351839829798, -1.3677282865212246, -1.139696815504914, -1.3540406180532945, -1.5080565363770755, -1.519727303972291, -1.3134867902801892, -1.5498399140614092],
[0.14526480283891074, -0.0545285042752372, 0.33476004407516735, 0.39121750219433377, -0.03374065660797898, -0.15086061782683113, 0.08401231677457517, 0.13505271590171467, -0.10610265607558521, -0.03530155699791448, 0.17445479277582085, -0.039637680130585724, -0.17321843146913316, -0.02452277665105239, -0.11571902450963373, -0.1476801245366144, 0.04684245463174988, 0.10254757682103316, -0.29809332248719217, -0.2269962802807524, -0.23809209150620927, -0.2975284119817178, -0.4601781540574663, -0.3348046811727048, -0.2008712800625889, -0.35310871571594343, -0.43738926350302476, -0.2906310517031112, -0.633966925498942, -0.1817492109700602, -0.4659721449823532, -0.5542465659322846, -0.3945500485959715, -0.1795164754422282, -0.6034059386338934, -2.095394773238216, -1.6828568063385527, -1.799354478432041, -2.098023649494526, -2.020419189715587, -1.9982237590800647, -1.8574066367462636],
[-0.07382605126806092, -0.04231728235136375, -0.4034123441635899, -0.3748994241944943, 0.26250974961741497, -0.051103139634580706, 0.16107589145650852, -0.25268138644544946, -0.25476639614755003, -0.41654326444171685, -0.29006072268770694, -0.019456951339422664, -0.13906696070742924, 0.043268600478024714, -0.27221807783667124, -0.312225736676904, -0.1360364305831533, -0.19427256217088418, 0.09701763262743991, -0.25351437712977243, 0.03505218474616996, -0.019908312761693976, -0.2235854732706309, -0.17103136007301184, -0.48046718972801605, 0.02241819527007838, -0.24653337628878355, -0.13531925518460225, 0.012435209102060323, -0.724208700479322, -0.010598623424916248, -0.19897089771493226, -0.44016548513654624, -0.7375505949482217, -0.4168059086979254, -0.9330959787613171, -0.6021130755031383, -0.6566090349277873, -1.4964420356174593, -0.7063313164647517, -0.7334281184824122, -1.684058740184331],
[-1.75515832187717, 1.4607749567054271, 0.6517634396938232, 0.0007548893903647639, 0.5884411205564143, 1.1978417812827449, 2.334430968226296, -0.8535606223467875, -0.31801752924653826, 0.14313490615516997, -0.12635857790896124, -0.39928478086937147, 0.37226249018709684, 0.9292034019824799, -0.09206644045084436, -0.4704926506279224, -0.2685098390607096, -1.1734440276515954, -0.25899290353100224, -0.14336746539549058, 0.0009584618327996825, 0.47025959507747545, -0.28266556562166395, -0.4139951045226136, -1.2318398299441984, -0.7760085200833701, 0.2602021014044231, 0.04201860269129431, -0.6639844217031596, -0.6600948847204916, -0.9530909311550319, -0.5641144345703417, -0.9206332411592372, -1.2829997806135174, -1.0183524112627536, -4.217927674975928, -3.768064434585963, -4.074108360794878, -4.11467978044772, -4.039310610319494, -3.837985181721694, -4.008745130388019],
[0.02095957068619997, -0.06013766389203641, 0.33855144339524595, 0.06746475457534072, -0.28390446248065543, 0.11198933712507751, -0.5250313137234294, 0.16828935735977524, -0.0616756559864506, 0.16376251769548344, 0.48558136933037743, -0.8525525521315596, 0.5210525540355149, -1.464070860155038, -0.28606320308732436, -0.44353326745351834, -0.5884456773572461, 0.2411331834117998, -1.0592303651359016, -0.3417764289744204, -0.6969479512272191, -0.49138934544420976, -0.37960221520306175, -0.6617968790031586, -0.6632473524007726, -1.003192117364226, 0.4738572656009205, -0.5625562911903564, -0.5453094598880031, -0.28841511796515185, -0.43213997862582065, -0.7976434847211041, -1.6681409949998738, -0.1557215943448605, -0.9379913672523753, -2.745347997472898, -2.220686580189934, -2.9204080043934098, -2.1520279112171714, -2.5598162899801142, -1.802439326787199, -1.8593454032755496],
[-0.3521449567287281, -0.23945086809360908, 0.02741914739017275, -0.251123975330998, 0.5230684531293781, 0.014490110194461158, -0.370922908882794, -0.036099035421116935, 0.1620267545245951, -0.08663208247094681, -0.11106390068879697, 0.3114432297296608, 0.1583791864694019, -0.10001946895640287, -0.3632966413653924, -0.0521011279212678, -0.15750964063562314, -0.18003809411839444, 0.18941660810524982, -0.14550511574631406, -0.18908707051497417, -0.3292811739508378, -0.05992498994100506, -0.22452248237488184, -0.30425117865918466, -0.09880191504110777, -0.1309924450600799, -0.19009210374107452, -0.4317167561662477, -0.3553848154933186, -0.3779045477532461, 0.09953627041804713, -0.6123303720057726, -0.49912042355329184, -0.5624532423037639, -0.9268580791527947, -1.1121951135026311, -0.3814834372757434, -2.1360403318960683, -1.4492178918616971, -0.6620822764893015, -1.5765230125200143],
[-0.058452664651324426, -0.06470287705246502, -0.14500732256983312, -0.25203973379085337, 0.0377808459026233, -0.033096668916650494, -0.01600914468323933, -0.17740246729034506, -0.1493399784319572, -0.11724157092101692, -0.3001076008076963, -0.007452093562371415, 0.009213862764731547, 0.05413269870414449, -0.2548182523903182, -0.27222023927260014, 0.04711195064481913, -0.3650399419167339, -0.09007486622547352, -0.27029331518228095, -0.24464949951176998, -0.2547098033870838, -0.18983986621093413, -0.15883793052282882, -0.3855552123223721, -0.22641471633831342, -0.038235991217483035, -0.2553845117145108, -0.4491347248760267, -0.507600041445003, -0.45256026614582207, -0.08785266284982027, -0.574317502396582, -0.6879974912582304, -0.3829740823235428, -1.2527341214805312, -0.788519083063699, -1.0696881480302955, -1.9700140847381358, -1.2255502581523656, -0.8859451250397217, -1.4339130570130403],
[-0.09604633521115238, 0.07556333754586184, 0.02207325099336286, 0.27140052524859676, 0.15879929085346212, 0.0067243573131562815, -0.004674832200451748, -0.10180777740912714, 0.20933407387891614, -0.2810968273675339, -0.013742915480500759, 0.22411602547307546, 0.19858486039290468, -0.2735326270603641, -0.10710930697488259, 0.0781196765466029, -0.24591223555328268, -0.35051472881836815, -0.2826199316129046, -0.37653723266803, -0.08033300275472294, -0.16548142059771287, -0.05517794666386909, -0.25076083574209845, -0.3724063339167498, -0.2756193386949852, -0.42454613464266816, -0.13248261983920592, -0.6829170838047162, -0.4511992712687185, -0.7164550920923495, -0.4237104315584464, -0.5140318692362021, -0.5848917648895928, -0.607086714975448, -2.1433229929060373, -1.7446820093480981, -2.3157901732325943, -2.154250541224422, -1.891707214206854, -1.9410082773876183, -2.1414308642694992],
[0.3732770138392405, 15.725916430971449, -1.198253715063106, 1.3678090880129037, -0.7269535598699441, -1.1748401699597355, 0.8305713646323264, -0.5602520688262627, 2.8008438005650707, 0.43134886499960107, 0.6175867013661918, -0.3836507166369061, 1.1346919367867192, -0.36822179822879797, -0.16022244507112768, 1.5912434980002768, 2.3324483230263096, 1.0722894697998664, -0.7113127280011864, 0.7544581126680184, 0.21741831288043162, 2.2783402981733776, -2.6220065700070836, 1.3431867368613724, -1.5580729901463792, 0.9625833855093122, -0.4649196263127378, -0.7303026045340282, 1.8282694445611154, -5.619058962530935, -0.2692745413368545, 2.7141130007349377, 0.2509530715196994, 1.6619472491603355, -0.7193813130678247, 1.9312468769123379, -6.075203021661724, -0.3894153224466484, 0.13416215856577018, -0.4807091598633493, -0.8982054083086259, -0.47186445904337404],
[0.22503227928026165, -0.24074426339228466, -0.4225001837757535, -0.1790317441130616, -0.11196946956697203, -0.2600322015406641, 0.26737749556162127, 0.18082021139902613, -0.23212019229119987, -0.4899457916437121, -0.6576515604924711, -0.27485391433189105, -0.6594351883861932, 0.10538818723702531, -0.24200396950803924, -0.3166459089310846, -0.3543764907633365, -0.6589840224706225, 0.17278565011285854, -0.18671905949091266, -0.4081611705907309, -0.585408624748397, -0.8166428748886858, -0.3487518672096678, -0.60577755548229, -0.44589342950535193, -0.8267335803838133, -0.5727911197291582, -0.610058377190458, -0.7713783179898838, -0.4415380105594589, -0.7213551161620184, -0.5064500144314658, -0.19697477898959626, -0.7706436171101071, -2.8922709113850815, -2.150837378332374, -2.460622062993499, -2.5867951602051886, -2.2007472086422837, -2.2442194269819127, -2.92327029069987],
[-0.04429878564186331, -0.24856846685892806, 0.029189542910507224, -0.09505424684282979, 0.2589476653193234, 0.15418211109699964, 0.031660181662875604, 0.08355240187710455, -0.15130700593372978, -0.23298241698692973, -0.39440554840260106, -0.0010438013332447242, 0.2400223010135582, 0.22992035801474747, 0.09988398228803264, -0.5687275056216726, 0.8396224854462386, -0.6776361516607324, -0.04841730919979992, -0.7442596209884685, -0.2179492332963332, -0.31952227177642567, -1.6257988983865026, 0.6296302310172879, -0.9667038347792328, -0.4943949354328362, -1.1377190755160334, 0.12137025882080751, -0.847822440639472, -0.5291676526082237, -0.5403200475400807, -0.8249635985834419, -0.6620131894252423, -1.1424814331482787, -0.8020916691204416, -3.373333294422899, -3.022187842389267, -3.2098095593273057, -3.738119301022201, -3.222525771826186, -3.145795544807479, -3.266611408124505],
[-4.673494924052404, 6.794208801551214, 2.683780103563824, 1.8026162342306116, 2.5954798580192713, -2.018286184666132, -1.1035631820772622, -2.19710621136744, 0.7094566761460299, -0.6615989555572181, 0.20505708853211968, 0.24847003813658022, -0.17421256870904703, -1.736010069328295, -3.2176158963531125, 1.3313985072573145, -1.3243944658689766, -1.3717453233822121, -0.7560151434936526, -1.5754092630668666, -0.6088327744314229, 0.09404827564585946, -1.4264528571413726, 1.2935676991406109, -1.1546720911463038, -1.3056453638256245, -0.6053612243741266, 0.6725238587528437, 3.9176717869764537, -3.92417928309709, -1.4883809403621502, -0.23457603570849475, -1.531801590073301, -0.840489482167839, -0.5202774084026006, -0.9641449204026912, -2.828891457398795, -2.6393709391281486, -3.0509868051574163, -2.4926436544568693, -2.7501089469846773, -2.673878454515785],
[-2.714511345601054, 1.209365792193305, 4.563325481733012, 0.19400833456913355, 0.7963742606456992, 2.532766515987228, -2.4271746153180302, -1.13186617051102, -0.5214757042141731, 1.8063446310456095, -0.8046495625562383, -0.46225339661179304, 0.3327071761919275, -1.0022108901115325, -0.4746267630867156, 0.3416602880410878, 0.445487879625353, -0.25182485267794713, 0.5624038230038151, -0.2860814597456586, -0.2517566961650685, -0.31595813580681087, -0.4085595361947453, -0.43909332508934407, 0.20207818543650893, 0.18447619802636134, -1.9261972370235263, 0.04622703101354826, -0.6603921374894519, -1.9344443224026502, -1.764642595287342, -0.02807681270974771, -0.6750221417465033, -3.0336364176439385, -0.5900909088022418, -1.823302516491533, -1.8501049194483434, -3.007128139123376, -3.671506985644053, -2.7610397015932233, -2.7875112854182835, -2.402156495778638],
[0.17111202951604618, -0.3088619480217017, 0.011025806285391642, 0.43608480201203864, -0.008880997686901024, -0.19208354582845194, 0.1531557764502304, -0.025867444391290045, -0.327260761383204, -0.07435012718069413, 0.1284990472515695, -0.008268392294796737, -0.09357243192723126, 0.050123683385050445, -0.027987541424582475, 0.007193424844867323, -0.18391473633066258, 0.09299629849803008, -0.3480208392066269, -0.23156482070877757, -0.1570792856121801, -0.4052215105016971, -0.407240609056124, -0.24841208803521403, -0.1876061728048541, -0.32897408859666044, -0.6531665307261345, -0.2905185956707563, -0.4807341347991371, -0.3345229580008008, -0.2815320045195842, -0.553203330621534, -0.33398651440901267, -0.19223200890265985, -0.5511468818568344, -1.5732909355630154, -1.226980701918296, -1.3739019849725884, -0.8500795492314659, -1.580057092212803, -1.7052852433755914, -1.7237453973227101],
[0.6166354036649603, -0.20137885184431978, -0.36462983925621767, 0.2551141569238575, 0.4526214494232443, -0.10028316007216626, -0.2687577611822337, 0.46954543372549074, -0.20361165180233734, -0.5488749939161031, 0.16156909411103895, 0.1176301748167048, -0.23342166519194177, -0.23285411204716286, 0.07112408111815635, -0.18898944957992467, -0.3182024453750378, -0.006469318154071129, 0.0204398798699284, -0.007761249573599897, -0.3374187212934838, -0.2161513309309552, -0.5042250537062684, -0.29821724731333277, -0.43613550512460575, -0.4333788342492742, -0.33886822478346484, -0.12762611988176115, -0.5525466120373692, -0.5319906245846342, -0.0601137955562959, -0.5818120828906945, -0.4814186734851383, -0.39446661732255056, -0.6941002922761589, -2.3610469472691293, -1.935722638156045, -1.7546218661672313, -1.657466808454079, -2.3334992720021392, -1.9420448485437127, -2.201011188359682],
[-0.9762968325498078, -1.6063909136335834, -1.2019377267306344, -1.2405971050842686, -0.5372658683548337, 0.2982819592011095, 0.24003319709192786, -0.719336495775959, 0.009659677830491011, -2.1934225299653334, -0.6246609895646127, -0.34299103908437567, -0.11853352916143492, 0.05495914482788877, 0.0359454498146215, 0.5407176956395262, -0.722503609549143, 1.489540040142214, -0.5161053605723227, -0.26182306790543863, -0.8525232341154113, -0.545921106694942, 2.2683120082976935, 1.9014552353464043, -1.4358023213568278, -0.10131698937343539, -0.9717108521477628, 0.09207869677039132, -0.5997895525682756, 0.9523388617274412, -1.3543657877080355, 0.49098851487607026, 0.4685487212972064, -1.707363884107214, -1.5611816265853826, -0.48358718315135724, -0.5156173341221606, -0.8508213341050632, 0.05991468523188276, -3.778965365871524, -3.002779000331931, -2.188161980118241],
[-0.5590236704463857, 0.5217432118693973, 1.6293663493930146, -0.4983735990316814, -0.6116063879454214, -0.06412878477905408, -0.145740273436359, -0.3218319204307991, -0.17943185480133061, -0.5287244919578767, -1.0738059389725367, -0.10955894147351855, 0.0909551129987969, -0.0694472750916585, -0.4505531688952534, -0.0848614097599648, -0.8926386831428016, -0.7209932353171468, -0.34021398013170423, -0.6037672718672534, -0.19787589167761935, -0.48695985012557186, -0.6675465649914781, -0.9264703570042757, -0.29141294899637177, -0.4986880906973021, -0.125151523750763, 0.04069220082267271, -0.9160892060145012, -0.49020056983376875, -0.8063090200090193, -0.04520778206277622, -0.059036741484398615, -0.4127349008174294, -1.0022163425951744, -4.299347622791439, -3.9639844079724673, -4.374861467131016, -4.3666437402331795, -4.443047546142418, -4.180996047247751, -4.362314432096318],
[0.16237937810425115, -0.15132190995228803, 0.2815395099489342, 0.3318700981899957, 0.33118333059423316, -0.19427146996072117, -0.20690407225404261, 0.15607695226338136, -0.3085612603737431, 0.05570284412164603, -0.18633491297754276, 0.1522223607856752, -0.3133626320501628, -0.09677008236105718, -0.18145746613842742, -0.7053609810813791, -0.03509869450673757, -0.0948131362092502, -0.22468221972810126, -0.15069326044563733, 0.16544469910850537, -0.27794551284561464, -0.9686530759913473, -0.35626136494749855, -0.6648625338031277, 0.05368878779608988, -0.7717665590674172, -0.20792367173706316, -0.9749392963009431, -0.6463834849456902, -0.5049043824477635, -0.5666023749872561, -0.4410056650707676, -0.4135104453120718, -0.7713323649110813, -2.8061320120462923, -2.3257280271043563, -2.789856164161096, -2.737163092951002, -2.7912592878400315, -2.3953005393219744, -2.622385644344352],
[-0.6991470619590491, -0.025991827056992798, -0.2687994046144002, -0.1476264683210056, 0.11746801773000895, 1.030406685865074, -0.2579224140504001, -0.5384251476134242, -0.0936398687119135, -0.37750818852247153, 0.4095850472776146, -0.2642246290688643, 0.5280817691954084, -0.03824609672316731, -0.20988087187787785, 1.306673950453972, 0.0004111058651099008, 0.8156371100678008, -0.3591246648396943, 0.3749285683490961, -0.10157460171042404, -0.5878431294258537, 0.5664523221787827, -0.19559725042630077, 1.30018677224547, -0.8073297824168382, -0.2763098867803805, -0.2631838964658713, -2.6390075858773665, -0.6080101760443694, -1.0090442367839398, -1.1413485799255458, -0.3957675434919107, -0.7719353048640994, -0.8627569212894448, 1.1561952119869856, -1.891078101352692, 0.24424282829491428, -1.0188569433206354, -3.019459196631514, -2.449042756951432, -1.1698248798524198],
[-2.241220405744418, -1.020268369007875, -3.9132402716063748, 0.5536255803074618, 3.230342934051394, 5.214325636763689, 7.806600324427212, 0.2765902806035271, -1.0479020768046579, -0.7707013807589843, 1.279834968955318, 0.9275136141789557, 2.358677940912391, 3.5384113087801548, -0.45335968202908794, -1.1227320169656343, 0.6609102334141145, 1.1920034871095329, 2.027319675167618, 1.5247979466750992, 1.5870128428069241, 1.9560280149323883, 0.3632174995155504, 0.40732131898068874, -0.2722466974628522, -0.7998209430338471, 1.6994052830516002, -0.42326061077795446, 2.924048284100336, -0.34863836907810847, 2.6254689412290584, -0.25878003362738394, -0.6293486461596839, -1.9705385129595376, -1.2765803392970854, 2.6134856255642287, -2.2510524175727005, -0.8507961617586878, -4.9330087728011165, -3.2126982559767896, -2.5396194282340905, -1.8576508888307777],
[-0.019807527642475276, -0.11415793691933154, -0.11181333578219611, 0.11325756614322026, -0.0009727909808175498, -0.12647821712487198, 0.23630470220080227, -0.11023400792659477, -0.22371577689900604, -0.10724164886728617, -0.0853548255928459, 0.07180669236114859, -0.15571119711685297, 0.22143417860655967, 0.002529021549161668, -0.07331834640457928, 0.261046517919615, 0.0020788171620835035, 0.18165080606905382, -0.05582255053468485, -0.1551027832264299, -0.22994028785695014, -0.3260220884321532, 0.1276231393590519, -0.46448463937777723, -0.16240876812109564, -0.42874781025304687, -0.11584186106947225, -1.013399115707399, -0.04879589802864124, 0.03869979301571086, -0.6120497697485182, -0.4822986228562003, -0.2449431151007201, -0.37957846739802936, 0.11233615260477464, 0.16271809663083356, -0.5263610089375462, -1.152552373057229, -0.9413813575434141, -1.4802139712076385, -1.643756295690793],
[0.09929797586224985, -0.32112221814152925, 0.11668432297016869, 0.39762159662813445, -0.318612949624187, 0.0875329353160645, 0.2518911563355575, 0.21135040254014986, -0.4053666178090404, -0.16667457429884855, -0.01893308630079109, -0.3344752943650251, -0.027885279891543296, 0.38522514194746077, -0.06112852271071178, -0.392227822929314, -0.10820076212057868, -0.03958385410242583, 0.00594031526154504, -0.2439588971465956, -0.011351325676570722, -0.5620762472139194, -0.3782677720259842, -0.17992099534399136, -0.10790491363976651, -0.5634744116422125, -0.16822828444740412, -0.358510916093425, -0.3320543813751501, -0.3756440738148832, -0.25681423644096735, -0.3851213217051314, 0.013403298278493967, -0.6185145575667276, -0.5861892909381727, -1.5887493887928248, -0.8776151509604702, -0.6778147008354061, -1.4497075505741523, -1.1660183149977592, -1.328091134873811, -1.103545623544987],
[0.3338961076774624, 0.015683907518298446, 0.3927270357122398, 0.2757392283147375, -0.03153315292439125, 0.07171740705054183, 0.1684972985901707, 0.31844287558855777, 0.003957607452559693, 0.40987447762801354, 0.20773941505834154, 0.18352829680674346, 0.10884435742938266, 0.09460112202612628, 0.1149624243838254, 0.1401046215276004, 0.40059131603190945, 0.502418047984553, 0.24731700417226074, 0.046794582498959084, 0.060991554830254, -0.021412514079210576, -0.4279098370448915, 0.8620283852573036, 0.5031012746634684, 0.21666331871740702, 0.07538054654278582, 0.05156811447104626, 0.055555852463016364, 0.87162630525387, 0.6241010528568174, 0.17604424002723784, 0.5505869232445958, 0.36360700973420934, 0.2605878006412978, -0.6564373798116557, 0.09115867693946088, 0.7476049594278735, -0.3449705008969887, 0.10431203253986472, 0.022472891601163694, -0.3672646237753108],
[-0.022099686312801826, -0.1106700732205772, -0.020486212510999406, -0.14539883577925217, -0.12988072618459712, 0.09776312506245585, 0.07343438992072822, 0.09832731230509024, -0.10249924307235514, -0.49798453075393406, -0.35807118823244344, -0.04203920834842646, 0.05002854495605662, -0.12795618909549186, -0.19522336431046852, -0.2798998848288397, -0.20116348149452448, -0.7409412963576272, -0.2440650355344621, -0.11406276600557154, -0.24242304135356926, -0.26526236771860096, -0.2882093780978116, -0.41684657639457584, -0.6678410410226776, -0.22748309380040355, -0.29716346525555126, -0.1850255238519862, -0.5389799550029303, -0.6520351313936074, -0.6336922990349335, -0.5746384373120178, -0.40963771232537016, -0.6860951749214492, -0.698169956213061, -2.749039198374469, -2.2904813642061286, -2.6903207863913807, -2.7537404784416046, -2.7476459558350097, -2.3143778955826004, -2.714949591481753],
[0.07229020129004811, -0.019063616013647366, -0.13926115241088616, 0.2030584771278814, -0.007864259479063144, -0.17578604039891224, -0.05962736214627579, -0.10948067669246449, -0.050611877686642284, -0.23871142522591451, -0.03663796673054215, 0.09358105354578634, -0.0978106369005348, -0.05601008187250382, 0.0564265211100654, -0.13502986591116062, -0.5797105626690221, -0.06261718480952935, -0.15629024721534726, -0.17484233570513738, -0.33846028553938784, -0.28866609752209843, -0.3537479306880096, -0.24315974410516733, -0.19004032804911972, -0.4213522010696292, -0.5241398510010383, -0.38860739938122796, -0.2834534226903044, -0.1799403504107147, -0.49247656984428656, -0.43099952579995376, -0.23174800877688892, -0.25343958629442453, -0.28848911456458237, -0.6773453564600362, -0.5586251203319798, -0.16893343273013445, -0.878785311682143, -0.5979851155955839, -0.9197361594913536, -0.6261239837511225],
[0.010589851776949464, 0.052665777518469646, 0.05305608004222978, 0.04180574535329491, -0.02209935369873509, 0.0733416891620386, -0.13490899701276532, 0.07853621721792188, -0.06670474112940648, -0.21206991455691368, -0.06602431717526165, 0.09027996403392992, -0.23232914188604661, -0.11554635263704183, -0.05726769533418646, -0.13919905505199343, -0.25013096937091694, -0.017667364373805016, -0.003440691261866033, 0.06271750378846153, -0.018874672073266235, -0.21727638187970344, -0.13442843561228637, -0.20168703409352895, -0.6271013835591894, -0.16896188589047723, -0.21657937364184587, -0.24257568332156992, -0.5845386171089876, 0.330735285609604, -0.33514517774686114, -0.6814066382390233, -0.3251030061040067, 0.17425461570358738, -0.5269962945823438, -0.36746002199851835, -0.08010829349697343, -0.9671004671256863, -0.5799375704638743, -0.8853019816498175, -1.046106357204338, -1.1923969444127231],
[-0.15428125472532336, 0.26610028501469984, 0.08164579425974328, 0.028361835024261435, -0.13858372631917376, 0.016453834632413494, 0.20324700568021792, -0.046684231906217906, -0.15213172266546904, -0.06168093462391998, 0.07259382645885541, -0.026169705389205652, -0.30725236488105584, 0.15776678220671905, -0.2436791699323647, -0.049420951004949115, -0.13372776417282897, 0.15454111227358946, -0.3203676669894523, -0.2830625632882081, -0.1187237059402841, -0.4066753454429399, -0.23604028194811888, -0.14490935656588558, -0.13630087768911273, -0.17755670485267233, -0.5410676137091166, -0.2259002700079449, -0.7892785879367817, -0.4424044904731202, -0.33678226478495077, -0.4396268737810151, -0.5497926985432121, -0.5165897911728792, -0.756063716253766, -2.774104960121779, -2.5110909413157954, -2.946502024583473, -2.5986859644815654, -3.0689107723537075, -2.804461442503349, -3.062105262378982],
[2.4008708502462546, 3.2473896464297787, -2.4005266599725696, -1.8232985229697727, 5.6043639167451405, 3.6869768922421415, -0.9674369744536985, -0.21345510243014104, 2.231909841062153, -2.539135560282589, -4.004773127602762, 2.5485160106207214, 1.5703527244872928, -1.1004997351517556, 2.6228492057126274, 0.1060092480943702, -1.4185796318812511, -1.217610953201152, 2.51544799390101, 1.5021902491889139, -0.590584215731608, 1.3359983321761877, 2.9907470847395525, -0.03910940557396953, -0.573097362589214, 1.8503735521024485, 0.039699941978870654, 0.2199065269878352, 1.8595596297992558, -0.5733945779392583, -0.06461382356767828, -2.489335198941177, 1.397052363345165, -0.6732820383262006, -0.5634945139820029, -1.8694850357213448, -2.146202501423711, -2.678459421950294, -0.843115867104896, -2.3341248959079355, -1.7952305838609366, -2.6001027629627758],
[18.778855313914832, 1.2293381181513925, -0.21112256088817857, 0.43158928845145483, -0.1918906984415665, 0.17574085061901218, -0.5876735050402191, 3.2046794567415327, 0.3171885919119403, 0.07507181737579711, 1.4872446936317043, 0.36806402173199537, 0.4937794019513054, -0.5724151719681537, 0.24041188209656517, 0.1955776244975368, 0.13124158869660674, -0.6074414527076676, 1.1999509442748215, -0.2028947164608228, 0.2347467864635334, -0.16538038937553592, -0.18757669850747954, -0.6982788908633754, -1.0979121499498519, -1.6960678157417246, 0.07598660764921415, -0.44184193743975064, -0.43475313121924436, -0.8922343690716051, 1.553086249537422, -1.4204574879130154, -0.3838801961719422, -1.124594128473371, 0.6289399872442097, -2.566563378843508, -1.9401172461320955, -2.2968863754937248, -2.1996214624502595, 0.5251521293378346, -1.2510413591891654, -0.8227630640460339],
[0.15655396137359665, 0.04875523260045275, -0.019057563854824527, 0.08122053455034162, 0.34005173584649717, -0.042594420397413194, -0.35384211664560644, 0.18373891722155378, 0.003585786156615132, -0.3260793179692121, 0.3441255167664634, -0.23269311737960988, -0.012078075612904674, -0.42406484559843205, 0.0068965815878483236, -0.17107088633152248, -0.3653767184639178, -0.07217840647637927, -0.5208845023428392, -1.0714527864237364, 0.10084739068391707, -0.29151141091347654, -0.6396520369327314, -0.20718822023904046, -0.5811612999267883, -0.41745581319206415, 1.3674193797605527, -0.04533148344580396, -0.4269599598962735, -0.735304417547707, -0.4565057191738842, -1.3549797466498221, -0.4326077071196643, 2.061456666006027, -0.16821283972804751, -0.7008285250796453, -1.5322129150055495, -1.1609925495434974, -2.0118306178037426, -1.1559452892663626, 0.25976521196703234, -0.7981454265732565],
[0.14519481010675425, 0.013082428860601767, -0.23324442811548002, -0.17428669923871895, -0.164771397501916, 0.13655201663277522, -0.013363543964498095, 0.1385239885123698, -0.17160998406783598, -0.18367468413263838, -0.10927829288419065, -0.10764843785181219, -0.08111012343985403, 0.04916650150593924, -0.46806484830439676, -0.3253689692467676, -0.26280931535193697, -0.1790150037469047, -0.21992191678497572, -0.13056364932101147, -0.15636928258255658, 0.04212663961557478, -0.48719824946081014, -0.3369207530587706, -0.15306072712611535, -0.5114697438562034, -0.5315884448833775, -0.11930132565376578, -0.44351431962868904, -0.5004434267011654, -0.1983150868133201, -0.26337079136211816, -0.3819736480005999, -0.6982040491556679, -0.3725389676917827, -1.1468802185061644, -0.9513291799339033, -0.9860770175157577, -1.229204206776856, -1.2103029510996592, -0.9066970888183332, -1.3576530042617132],
[-0.0019752233740109875, 0.438238457040114, 0.008192212446781227, 0.4535252401511255, 0.2718388521939639, 0.22898771263552653, -0.1438633067914361, 0.08735580029797797, 0.1970686806279674, -0.10391043074768729, 0.06174871353784658, 0.006378367213981241, 0.02041649661607518, -0.19352218558935397, -0.19606457525321425, -0.05173510781565291, -0.32815982929326776, 0.2713088646850812, -0.05773659675229632, -0.08549582206291499, -0.09932219733630376, -0.1666152250422577, -0.2583813276493741, -0.06398416463287493, -0.021116488877112986, -0.264370971232243, -0.2030246148259449, -0.26821177727618306, -0.7570694966383953, -0.6475581862666083, -0.32626118492320483, -0.3760981503850984, -0.44186788260229526, -0.44300333849211027, -0.852315612654029, -3.4260427034415355, -3.0639128556446136, -3.517612971294833, -3.3960123319664266, -3.597066358018193, -3.333073562766969, -3.461318677161222],
[-0.20431228341666086, 0.07407421805064215, 0.05909015584820395, -0.10427411549822924, 0.16378449859470723, -0.03767240061750668, 0.11396983138010237, -0.30617364554535287, -0.06220658632047882, 0.1459707790864923, -0.3726214676072285, -0.12917197935311803, -0.0470367409931102, 0.16201785837668656, 0.10498886168174933, -0.17191547992309747, -0.05871518625667721, -0.36693929845843304, -0.34603858861077685, -0.16942595371827743, -0.2436445605885482, -0.042876792835427203, -0.0062468651250177165, -0.2352499988432272, -0.21070668597310946, -0.34969995432505446, -0.454115292166024, -0.45225189970034957, -0.47377841455407926, -0.4143278749355651, -0.30496661776312994, -0.2484686289060987, -0.4213287074082847, -0.7576674087682236, -0.39133054806982187, -1.099305757379632, -1.1577184076716691, -1.5211418238247336, -1.8157678622899576, -1.3772425454850377, -1.3530344592854882, -1.583597441073279],
[-0.06712763272057054, 0.12201975158496414, -0.3639248051148623, -0.11468851312504343, -0.2261517289266024, -0.16777523152101267, -0.004569017759386876, -0.04675436321869637, -0.09330884769083747, -0.28414235390076203, -0.2619232013956704, -0.29294454702130224, 0.027832954305846248, -0.09236454361430604, 0.21165478768315543, -0.1923409192190433, -0.43089997049625267, -0.35421593072022123, -0.3931201274775086, -0.32743833302720676, -0.49208921344660667, -0.1357445173009963, -0.31868691441037983, -0.682203610966974, -0.5421904690938579, -0.18273521959580552, -0.6056298854194524, -0.06147412717652624, -0.07356831658797866, -1.0918141008803441, -0.570671653041695, -0.14623347377633553, 0.22428391744311177, -0.9922322675715142, -0.49338412910233903, -1.8411860561446618, -0.4386244634597779, -1.4548176024796584, -1.4665989441760083, -0.7672425689600099, -1.1592763394895536, -1.066532574899265],
[0.26914893490879405, -0.2380225684158656, -0.6853344970264823, 0.1353147930639464, -0.05868862129781078, -0.45816572149471935, 0.20893281557274457, 0.19086150824556386, -0.5313252159708175, -0.8143285788924418, -2.1038976217488057, 1.9181291028797007, -0.7038362398397975, 0.6552450874899428, 0.33418330884454467, -0.8008318724933874, 1.245940641461886, 0.21015654732514757, 2.104604933457101, -0.0492956843313311, -0.5298850289436611, 0.2523902990199572, -1.0115103339650646, -1.378978801996893, -2.115462327004135, 1.5744211907877124, -4.993199988696358, -0.21781415330988316, -0.7151821968273626, 0.40274767809194867, -0.17004484187434216, -3.278142807337116, 1.2676029905987658, 0.5166067657365506, 0.5474812005625016, -1.7696703912373595, -2.823667878550504, -3.1096627777104944, -1.9468801458089555, -1.0909584880095469, 2.424987221463762, -1.0618181902326533],
[-0.19359969467917681, -0.062443254811919084, -0.37592987828653623, -0.5381003018634355, 0.007827025536234914, -0.12454861785746627, -0.1473741169247868, -0.26961346805507513, -1.2985507196708608, -0.06283760109880765, 0.02757617504521405, 0.3149582920799668, -0.5123389308586496, -0.09309838368472612, -0.31980531312308697, -0.029908464741029706, -0.24759677370306116, 0.529740488140087, 0.506468498558144, -0.437072065445725, -0.24408923252720374, 0.06457933463204237, -0.3942425647869326, -0.3493457249707809, -0.8041682728315628, -0.3910616797919754, -0.8883294640410175, -0.6868497016916066, -2.352066406792309, -0.2525985718124081, -0.5474930366294871, 0.54887946974511, -0.9430271971000057, -0.3874238717337011, -0.9575466927436195, 0.059900749549968255, 1.8306505454759232, -2.2706251504498, -0.5599008145876703, -2.136950376496283, -0.8525486693320106, -1.3386869499364502],
[0.375856758957369, -0.04732603722631368, 0.021898750082570306, -0.20636908165443824, 0.057725227820960026, -0.25287984108096756, 0.02371116450363314, 0.10212559216907632, -0.060609579509006604, 0.11964773967411368, -0.4476062629296751, -0.1871296244729145, -0.20531257354571797, 0.19106163733936055, -0.05222994466625285, -0.27114009413600837, 0.1771153621527492, -0.5984840179113478, -0.0044140765468828405, -0.4235501066649134, -0.10940349737605924, -0.6314415681592579, -0.6101404182689933, -0.22138638645856187, -0.7811077071590083, -0.15202668692154697, -0.5961463718004554, -0.04357425660397056, -0.8823586571089581, -0.25502788151342265, -0.4453541259773948, -0.35054925642933826, -0.464164489331461, -0.3939982111903005, -0.4442081058718106, -2.7012599651987954, -1.9870403750936008, -2.164239208519863, -2.6630061000397207, -2.022706767794447, -1.8421250863093854, -2.3455061475399064],
[-0.030710530042611487, -0.12487898534967726, 0.25047100612758544, -0.1389135880019234, 0.10764971457359998, 0.07221173247760465, -0.1831956951540276, -0.2804128710813501, -0.2562525608298664, -0.16137286461690958, -0.13218941892756786, 0.18876839351060165, 0.21618841435056768, -0.2033249512102816, -0.25002565547827305, 0.05841308247988283, 0.05453357705426616, -0.38073292442014006, -0.04653302853009755, -0.281514234073661, -0.10681733829563402, -0.6020088298633987, -0.12033845195985013, -0.16883736149745826, -0.49166896772776203, -0.3824857036934411, 0.0021600182400555975, -0.3565887721458492, -0.7521359942971869, 0.03461154655813451, -0.6509255217385611, -0.35213439530597296, -0.6006853023364767, -0.2917038419640657, -0.4320552872668991, -1.526963110127062, -1.3713151549210285, -1.5498172168890967, -2.0269166425094425, -1.641019637575899, -1.2246537239293867, -1.5569024260549773],
[0.7284146769109607, 0.20944678129946856, 0.42752820581233447, 0.5649024304404106, -0.36744489802068125, -0.420747762940607, -0.6147641352263763, 0.17437089459932284, 0.047738134740093544, -0.1992120496799312, 0.3347817319253397, 0.8209793393247509, -2.5703516166977534, -0.7099580361585452, 0.19239601884443475, -0.44614334381820614, -0.3071899137165655, 2.40707937910018, 0.9301754217108298, 2.415558469299861, -0.43328752561030753, -1.0877360044793551, -1.3253298870179004, -1.0995537550340748, -0.8795586250169303, -0.8020835371620861, -1.1869714824671038, -1.570766217188806, -0.6475324801268568, -0.5624604758007906, -1.1545445311814817, -0.8613408840290815, -1.4319349943878192, -0.7218883736853642, -2.0527108683697057, -0.8081263699251063, -3.245643020489998, -3.735016370929331, 0.9517836907014534, -2.528918073304669, 0.09294976024876789, 0.8766850923488172],
[-0.12280799697050428, -0.1268480381098143, -0.05577265708886903, -0.1994976604003446, 0.03984240587078844, -0.0011062351032140366, -0.17202187124952875, -0.27599588760400545, 0.1346516387604295, 0.17372964788371095, -0.011427670538371688, -0.1664690988088344, -0.07042174769691212, -0.0973979424710176, -0.1356619002294065, -0.08785755053521453, -0.04113369598377779, -0.317862009520869, 0.07875548627372984, -0.22577511928265961, -0.13146343782797026, -0.16763797647535886, -0.09829768552608735, -0.18036724021768422, -0.3240213497447809, -0.29229225861996405, -0.3100392045618243, -0.3869351932298427, -0.7076081338503489, -0.3326035758321765, -0.4305843527291608, -0.3754876635055537, -0.4761927225975288, -0.32974183975209576, -0.6617382986417345, -1.718707869175509, -1.596735526370133, -1.8537817467002198, -2.4008809282524934, -1.7720659683628663, -1.641658545623078, -1.806652816276762],
[-0.13773517503052204, 0.0867111136610605, -0.24096043827977567, 0.5434006653560718, -0.2003509487271135, -0.0073980651856713, -0.08732049633862275, 0.2300104782782617, -0.12161679679184502, -0.3739524379497104, 0.09333895681581382, -0.2172254994289256, 0.00197215744290826, 0.02145572366518297, -0.16040574404142885, -0.2610933540968891, -0.3041978499146538, 0.08819629812853445, -0.15809995677001495, -0.041430772881542906, -0.2560100739004482, -0.14146844931006913, -0.5485867563479447, -0.11819195441977065, -0.02049544907700174, -0.5429191751228463, -0.6601409376974703, -0.39272532141310657, -0.7217765403651704, -0.2630158860935292, -0.5109400051934646, -0.48242619819709087, -0.49262690301854684, -0.3429054005604545, -0.8311349915822779, -3.273887344644206, -2.975603705112051, -3.3010612988466477, -2.819824169287097, -3.3928252851205487, -3.1597066289588636, -3.3994012606177932],
[-0.31917122709013546, -2.1567448732631744, 14.260462883170526, -0.06520191660474293, 4.206819118158539, -1.4242310489883063, -3.677498018982608, 1.037063267692351, 0.8324045937317889, 5.841933110121954, 0.07562591237384082, 2.61377555454925, 1.5372374645445388, -2.760383229017477, -1.5527217046651542, -0.21944492866958268, 0.23829748239926477, 0.3647084113681593, 2.6973145816509754, -0.38250714365576166, 0.9410184335017403, 3.434272584145695, 0.7916356148043457, -1.5917456119748794, -0.4671283938783893, -0.27145040562112527, 2.530918927856247, 0.6671656287951717, 0.7703452171750284, 2.802526254809262, -3.656000590721914, 2.2614982593976682, -0.8913307730263491, -0.2453675521426591, 1.5569791033011116, 0.11328522814853301, -2.5447791990775266, -7.906769331626167, -2.5223309201867155, -3.3178547652976422, -1.2850619266008685, 0.8705496861726896],
[-0.09991974585517818, 0.04909165992051127, -0.12722310273496218, 0.11758120781151658, -0.09525199912599258, 0.13200164690559574, -0.09000917314679262, -0.20284986685260148, -0.029492877880863293, -0.2569076827870056, 0.4971243789430792, -0.27038313001605135, -0.0554294357027649, -0.22420231798972526, -0.2703495458819898, -0.2701290737407719, -0.031846811795995404, 0.20427223604537176, -0.39951654226853633, 0.027556115212233865, -0.32537137258637056, -0.18561880072804968, -0.5612939220848424, -0.15467172112746505, -0.11181891210738697, -0.5314361297071629, -0.10386189460716325, -0.20639366840597798, -0.30254198586079367, -0.5310709477975248, -0.317938873673905, -0.6356423891390941, -0.548177259936285, -0.21143757403600516, -0.4030615833873779, -0.43665278242828676, -0.9561300287921889, -1.1876297393603896, -1.0789591500372535, -1.4565609695774207, -1.1712624606779092, -0.6588959965423615],
[0.040589370852283806, 0.09301334571303667, 0.2122837398208061, -0.007668335367228786, 0.19410580491498924, -0.1113043320698292, -0.14380991464389267, 0.040577667310081854, 0.23452970279032523, 0.22868783365444212, 0.1699486996410198, 0.0942015272068196, 0.14493429802431512, -0.11127444079157724, 0.2224351060595668, 0.2687400537379781, 0.18661698049634493, 0.1332555210705999, 0.2224009374999702, -0.034324635030319114, 0.2739084816901352, 0.341375098977316, 0.4828744366537667, 0.32360901315017304, 0.03408807308339993, 0.3277446708439729, 0.321066249722002, -0.081793282574332, -0.007251664520285727, 0.6633288258284985, 0.1724604126004005, 0.2619457529677235, -0.02117352689197513, 0.2736761774440387, 0.052566060215952774, 0.35482626543674034, 0.5400985910262031, 0.008511498859665909, -0.7219567790002148, -0.05247852858943988, 0.18692473214218944, 0.02516890670571677],
[-0.14355335165926053, -0.023659770936052604, -0.36829841708400474, -0.6320966145769928, -0.5074148017905462, -0.04017126311443724, 1.0401123662846463, -0.07056562568739548, 1.99379928953911, -1.6509279857362624, -0.6407724234489296, -0.74609842198621, -1.3945090959518616, 0.23073755813415522, -0.8529892260324803, -0.19350690472933862, -2.1169348727381596, 3.275621905228111, -2.3302262659837965, 1.093656364712512, -0.004813851554345592, -0.399140364838129, -0.4964683566788402, -2.777004774145328, 0.42210529547563536, -0.6666011170500462, 0.32319886214178145, 0.08996604452125244, -1.6412893042431829, -2.468676351678444, 0.5336591324590593, -0.8207361884168944, -0.43521289896701176, -0.9654935338002749, 0.06113579885701396, -2.6850349661191975, 0.5256865196534822, -1.1830453103224976, -1.5976172438626746, -3.208052997096462, 2.178039025507473, -1.3946390933844273],
[-0.3075049489776265, -0.17473582466603296, -0.2568140387458126, -0.5833998671315355, -0.32839132069675847, 0.31277409953830154, 0.07512993149334783, -0.5968614860491731, -0.4968031157904213, -0.993110810766564, -1.7857726642169411, -0.7346659160939494, 0.6904930793061901, -0.2142770844828415, -0.806613226405677, -0.49177187166236686, 0.4315592527944061, -0.8449394572635561, -0.6864617443366905, 0.6751148971064406, -0.4208996170213983, -0.7554514986388281, -0.3195953971555181, 1.531066183998623, 0.9240673926438798, -0.10667001121231348, 0.9237208495138621, -0.29890814898127943, -1.1424111374208028, -1.4802132287600167, 2.3345594166872665, 0.6807055606340479, -0.34323370640795353, -1.0844480827127991, -0.7500275618551868, -0.9908054485522254, -0.7454077571474409, -2.1844771154569793, -2.233197430606112, -2.586369260181464, -1.6783388174511455, -2.068821817517292],
[-0.2947850363287624, 0.39962580784318963, -0.22956663921183362, 0.014482064813766996, -0.16305890058168943, -0.07063593140818777, 0.13587192475065613, -0.3578585326795244, 0.2953295174594043, 0.6867836493015461, 0.08408171645416193, -0.37450302220280673, -0.20028641240000786, 0.0734005635625387, -0.25910448624004756, -0.3692308943659196, -0.9488806255123241, -0.3389567960014983, -1.1198185606887123, -0.3333796289804715, -0.45007225270510187, -0.4867275775570464, -0.6106782265463641, 0.6166882338462896, -0.25787003465328395, -1.3412726993375137, -0.7471416052165568, -0.0006816455965613619, -0.4685611317012443, -1.2006073271175373, -0.442428893187974, 0.9884692983217894, 0.31040957939685265, -0.5192895077355237, -0.7567321254331713, -1.3623002820563637, -0.30447183883744966, -0.006983914069278628, -1.1740598027889957, 0.36551803275735795, -0.8155436838044335, -0.7975308971386457],
[0.10689734134673003, 0.0010846528593984447, -0.38444854514974985, 0.02407086073885475, -0.04911739732753759, 0.21003359623048953, -0.11575443358139832, 0.18601196689720045, -0.08558494470909263, -0.48332003894847686, 0.035820756526142264, -0.06565374089105477, -0.07789352181822397, -0.30575797913917263, -0.5599719710853146, -0.30443577827873025, -0.39643242896812964, -0.27404490632018086, -0.007616003217092015, -0.1348422447623586, -0.1190247324269148, 0.2548207994388252, -0.7474996408766637, 0.4987801884612451, -0.482039335347705, -0.05549551273992591, -0.2364149083581731, -0.34391130002605813, 0.8661280493080595, -0.47223952404864805, -0.20200314600707475, -0.607772451387459, -0.5030648784131251, -0.16004975867832347, -0.5106909401499131, -1.1947268985805, -1.2490627685057694, -0.2645654809026479, -0.9509846583783818, -0.3507601967059201, 0.7118093680221467, -1.0172117020479785],
[0.16660748303206963, -0.313600905397008, 0.20698301378491277, 0.3332643051254805, -0.17965785715557717, -0.24166784357887477, -0.15773303584214562, 0.170757295534848, -0.07396264678649428, 0.29220027670972626, -0.022989487732959807, -0.23595052843688108, -0.21860756153222727, -0.052594390756613325, -0.18355504986011864, -0.4540067446038844, 0.0417728615194723, 0.30570238108584663, -0.4280251158491488, -0.5040114656705997, 0.10496404845947088, -0.32379205405316713, -0.729794568746686, -0.08427010458284881, -0.5247422756574419, -0.6248203995875034, -0.7821602430025629, -0.2770169552360844, -0.8725699323680534, -0.41619941270931377, -0.45251986029654967, -0.11829079029838542, -0.6830432562397699, -0.24273507642450626, -0.5205326845320185, -2.1985183217220747, -1.306266814020042, -1.1810156319754952, -1.9227737394872695, -1.1642377053379376, -1.0654528295303727, -1.7311650628463922],
[-0.08187784254450328, -0.147973652625407, 0.08064918703418522, -0.10828986631490796, -0.0782184512277485, -0.008615078185194736, -0.0049596682087776735, -0.25211223543654926, -0.03174637757858446, -0.05482947379043317, -0.1787137570307674, 0.0006811593359792908, 0.0522982634899303, -0.054400782362838906, -0.0535607176299553, -0.07697376264337623, -0.08750501032327404, -0.33020945059814805, -0.024721688784275175, -0.061518745609356745, -0.20792762108754326, -0.2746526482653775, -0.29210468138889556, -0.06557830589395182, -0.32903754905766736, -0.25155365677013636, -0.20811752258088193, -0.305579096148234, -0.5172710484090253, -0.15297309055827754, -0.5262577832078513, -0.12990877182371727, -0.3353525964447443, -0.41572448705735, -0.5412032037172545, -0.8297804019404295, -0.7989783837658565, -0.422898174840972, -1.9906768331739106, -1.068583451728938, -0.7603530645661853, -1.0484538017249814],
[-0.41598116904259164, 0.14091839414574414, 0.032365784135043825, 0.04449006604913536, 0.0911625611842973, 0.049775146052195955, 0.12855471832605816, -0.11148990000423308, -0.007069444092723661, 0.11250006788499248, -0.0010176531383286917, -0.1427238776159451, -0.19251936097192224, -0.047088644825658464, -0.4187929697653803, 0.05562908809150056, -0.046533005726940656, -0.2084864637727638, -0.10496447038556211, -0.021254710796574292, -0.050567421408981524, -0.2328733427100392, -0.09903799713910233, -0.1697370449249316, -0.18670543127436107, -0.35555006471782025, -0.24681462562596346, -0.20794225606595013, -0.7493074422811267, -0.3498083743411381, -0.4797421172976569, -0.5600491818586273, -0.2732752577028177, -0.5892289832138388, -0.6935109837520826, -1.5047957370969915, -1.6747141092495854, -2.117583607337566, -2.3960142901152177, -2.1029212103781183, -1.955885671048535, -2.1727320519131283],
[-0.03160065325854583, -0.34053973063137954, 0.07069394602986877, -0.0671251774165756, 0.7861542090299884, 0.07524730318931575, 0.8292252019599538, -0.622579418357566, -0.1818600346099754, -0.36533817805478436, -0.8121630610851163, -0.17394412408313611, -0.5984988085609126, 0.531560458717102, -0.27851543090964875, -0.44003818686879054, -0.15316307840471116, -1.2928614938154714, 0.2927513392704186, -0.294686130671014, -0.30894210864166216, -0.5906801231157247, -0.8371078531506979, 0.04187882080732047, -0.5096754037526224, -0.5750733954318215, -0.5589329896128766, -0.7820795838147576, -0.7380699836367274, -0.43028914447193956, 0.13358014129057713, -0.4150475174595225, -0.32721618400773683, -0.2744442639017602, -1.0073973972217625, -4.042865818966194, -3.6084420994590753, -4.010756206816895, -3.8713888042797775, -4.063015839743674, -3.77080568987372, -3.976686071025393],
[0.09928460807575733, 0.058784301944254526, -0.008708506615754813, -0.15730373679306564, -0.21976768654868525, 0.016236351014451998, -0.1946604591623275, -0.05329004509870587, -0.24641221508982591, 0.03383126421783013, -0.32819875228543094, -0.11797804186531978, 0.11317907814108516, -0.14877926111727402, -0.09315415723414447, -0.26702998390173094, -0.08210379849383398, -0.29441057501097834, -0.15765786963859207, -0.26333699644877245, -0.2272691046796766, -0.21058506730183654, -0.2898409615386109, -0.2508781757686776, -0.48509076835153225, -0.25038890925613594, -0.12648433166278344, -0.28649900155786445, -0.7197818860801134, -0.564018581104854, -0.4988449458480281, -0.061134793751448285, -0.46852701213162284, -0.6981576919112612, -0.6827331417325603, -2.108799550229147, -1.7908510439533174, -1.8458382977274816, -2.708006543544391, -2.0423389603886983, -1.7997939082723153, -1.9982020023498284],
[0.4695311203065954, 0.13341286308784384, -0.2533902846065823, -0.044641978346361175, 0.1698466480544109, 0.14766639578652802, -0.19730458417410246, 0.3297267191357025, -0.016494410108966212, -0.6241267585758478, -0.40037309711637215, -0.13144466007306166, -0.47620754013094263, -0.26650554826133716, 0.10891145606234638, -0.014422752319262986, -0.9183027398904624, -0.5476496860403512, -0.014439155009513545, -0.42350854667060034, -0.40971596796244136, -0.510944342041182, -0.2244407562148779, -0.7822975841771385, -0.269225495181653, -0.5693786921123916, -1.1284083530940177, 6.107561072746631e-05, -0.6150693441439777, -0.6364773858658274, -0.51590350137053, -0.3767869539809972, -0.6542282773708809, -0.6006680382618171, -1.0323702160988788, -2.7962999786958433, -2.0922811068834957, -2.568897243280808, -2.3361554594002394, -2.476686370408975, -2.25753592968188, -2.166150537762718],
[-0.14716764967639315, -0.4274077913474552, 0.02388704866975392, 0.6733623182187268, 0.8417824060924921, 1.2918275782350133, -0.5946340197938366, -0.31787215205603825, -0.7984826834869267, -0.14771402006200082, 0.0020414169440062155, 0.18929513014946078, 0.6550692137329372, -0.33678316829617544, -0.2532615586342401, -0.9579568128524475, -0.5190007808280261, -0.07137897383023714, -0.28969096513853243, 0.6596437549137492, -0.005132756061077863, -0.41124844233597335, -0.3793136980274046, 0.1554227597136202, -0.5907322459881401, -0.7950410638553044, 0.19702317108349301, 0.264923414655786, -0.5260045477928762, -0.7219771996856991, -0.5796340306309585, -0.15710001308424387, -0.7796758254633901, -0.28019059054402823, -0.6563898889912501, -2.4266205882153984, -1.8898380319619401, -2.309195016769506, -2.9293832320167166, -2.8371365547176817, -2.1815043742379547, -2.2091061442860886],
[0.0756625551306643, 0.1545853899695135, 0.007773795909812031, 0.030177545621363037, -0.18543066215132353, -0.03391032993778687, -0.3699201929830884, -0.1526761271114478, -0.152883042623332, -0.22635579953430973, -0.05364511194594272, -0.17633987947071617, 0.14826257144964847, -0.48499204967221915, -0.23822449512222996, -0.3985405733000089, -0.06877452930443294, -0.2235623686163213, 0.28328815524382556, 0.07360815023348684, 0.17972293637468395, -0.3172606576210015, -0.4617785082201558, -0.6363157959552854, -0.3522511150469382, -0.24612450949809625, -0.2999027250747742, -0.15668933894276704, -0.502103686372694, -0.7419396282941395, -0.3979126438825285, -0.7727158416343894, -0.2626608801973785, -0.44450508700588903, -1.203296528188245, -1.8864183557790368, -1.255545504858382, -0.579757307436758, -1.0140303481647257, -0.18040394157879203, -1.0551587016106334, 0.30887231044173624],
[0.13460364482796583, -0.35294302001590766, 0.2702674515819427, 0.023608246248552647, -0.020132334076578783, 0.16067234387811222, -0.21811446127189807, 0.24156869057052852, -0.38206771973485754, 0.3180103904309748, 0.010060052325236596, 0.13533785931707545, 0.027734476639695044, -0.22660872443949345, -0.07960668829221566, -0.15577638287916398, 0.04322476562437784, -0.19160146453892618, 0.018967521010983167, -0.1926016722665214, -0.16291435445286154, -0.20093727647477547, -0.6666568161529103, 0.05338512340515619, -0.18798833974024703, -0.4138658534507829, 0.14306362654471785, -0.44462472294502686, -0.5823990292014215, 0.04125838578180563, -0.2566448968505922, -0.5323526447160084, -0.5234890300020476, 0.2548600752199742, -0.5276382243514514, -1.1424268963725137, -0.9627515896597929, -0.7355347241612047, -1.713485145367849, -0.7116457531126309, -0.760589787539177, -0.8943499703812071],
[0.04920374225073611, -0.03966764245136223, 0.11712303205990117, -0.045198558827139426, -0.33019531059516377, 0.38908649934208217, 0.1912749653366706, -0.0973547901417122, -0.37085750753168784, -0.041425891606158415, -0.13791806596140976, -0.3605932465788778, 0.14478657622817526, -0.13693867709896387, -0.3835415659211785, -0.394590924940751, -0.24956913084773819, -0.23826850153245602, -0.5074916185143373, -0.22238322570370367, -0.019293845097746478, 0.1224231962645514, -0.7723007872595292, -0.46373927934445897, -0.34483068162962, -0.7187988006668344, -0.0022208799592217388, 0.02135278848365321, -0.680957071431593, -0.791317426697693, -0.6147963632789427, -0.3702848708076324, -0.32805532208129706, -0.37239114784598426, -0.887505552529463, -2.2342090434887103, -2.0347748515702784, -1.8915202143914756, -2.4153406961554853, -2.012080614540643, -1.6676944633363169, -1.5626433677883647],
[-0.07835926098968628, -0.09891107439949319, 0.08534437961194155, 0.06609689469453696, -0.017464527103522667, 0.06280951013349294, 0.147249161392401, -0.07118478635012403, -0.25788330029905865, -0.0536671756472483, -0.22615525688797597, -0.17582370883679238, 0.016654460040451652, 0.1950696444853861, 0.03345138877634691, 0.08221769252797008, -0.220344853782776, -0.1661151594073424, -0.1821851749641519, -0.06278710783624461, -0.08612874069286011, -0.19792118831566233, 0.28685579590418425, 0.04612703799746968, -0.3420452225686175, -0.3229909613409441, -0.5244729577888044, -0.17556626756114987, -0.6330752152435445, 0.8230858279276992, -0.2794357714560409, -0.4304692923737556, -0.08630710586655506, -0.17359484810616763, -0.25729898451377, -0.21436944927378382, -0.017971253881357882, -0.7376153523285172, -1.0549102827768793, -0.7482072938886672, -0.9754469266384188, -1.0080279054450825],
[-0.14307790851600508, -0.1244073028078813, -0.21544331990072732, 0.052414282461365314, 0.31717184656691383, 0.25308192859754974, -0.37661360722525256, -0.06898494731850614, -0.4471530101448833, -0.6139914938542196, -0.09970724886901827, 0.12346168858166487, 0.21364578421691438, -0.6789553466566288, -0.20309073344364773, -0.36303149691079095, -0.285103503029632, 0.22637417841279506, 0.002327293796252999, 0.3665316741573684, -0.5967747495563008, -0.3973569979593585, -0.30009965533219995, -0.6221769011321543, -0.07471062280856502, -0.24473606800067438, -0.09767762439095257, -0.6795489422342171, -0.2655264776823217, -0.7337114912014554, -0.4559862764894729, -0.07053701950947919, -0.389218560875769, -0.054096819077285224, -0.606827329251635, -1.0731631278201301, -1.221354228977336, -1.09094552190337, -1.1825213791203066, -1.0263931684756769, -1.1935837109451826, -0.4843397900709336],
[-0.19384619035295111, 0.3723029598571085, 0.2190571508030989, 0.24424728696217196, 0.2871331268355351, 0.26140701830780055, -0.4378536354796128, -0.19778370222741215, -0.06594849536272859, 0.11415912155819734, 0.46201035977228927, 0.13916861017264961, -0.0004298305632392943, -0.4609889876792058, -0.24581971797832594, -0.0935168783776724, 0.05624940003800805, 0.125108758332194, -0.08843627720923755, 0.04778053327761917, 0.095205697938697, -0.05222013278140258, -0.26126377210341845, -0.13191556924791745, -0.24215032804996536, -0.43752385257038934, 0.007161381355327881, -0.003544376584547091, -0.7910275665430057, -0.2797329742913649, -0.6609953353819746, -0.36989800100925346, -0.34211959998722, -0.18595935174129952, -0.5065811875985538, -0.9228112473873489, -0.9360872834240102, -1.3363403477036981, -2.00690942908402, -0.4228392316110277, -1.1221043476890868, -0.046866337762648244],
[-0.14137755823291145, 0.5387118701194512, 0.35447532694456135, -0.3561773414133026, 0.3273105810985308, -0.3455547298378076, -0.31866180886693823, 0.01722127337291625, 0.044156284591386985, 0.000870270549233657, -0.15307819460612118, -0.00910733970849167, -0.08300475489333896, -0.5052829348682913, 0.20983317956977607, -0.1504734708247, -0.24433674606159345, -0.575168481484776, 0.037405064656347305, -0.1773499834703133, -0.3898117268148269, -0.18657745964501896, -0.10171168793290858, -0.6830314286817593, -0.37730346481226024, -0.15459433524240557, -0.5894219460143306, -1.074771964345394, -0.8681095379765401, -0.7386360634988185, -0.6924697203610848, -0.05498979776280412, -0.6262024868875956, -0.8086729111960361, -0.2554775342114813, -1.5332936456914172, -0.5680688291680596, -1.2671284707068862, -2.095520468718357, -0.5684209107036331, -1.402067560237762, -0.1591571493825509],
[1.3479567070943645, 1.8510638112477666, -2.1114182424747914, 1.2282545412410903, 0.6166506948184542, -1.0186747682541237, -0.9841168317101755, 0.016677234516000842, -1.0828035924185182, -2.726640816300126, 0.10968351715013999, -1.1394555099593446, -0.4981418431944771, 0.9859309141087523, 0.44829273368766837, -2.0359689202322717, -2.891569977052745, 1.4719073437867596, -1.7835871864673394, 2.234655107786234, -1.0973381787608203, -2.3242635087382495, -3.2841081373428183, -2.942109788854853, 1.671142315163165, -4.245932746749881, 1.5617224257857596, -0.5327298459865649, 0.3187610544835749, -2.849428189794561, 0.5273377311944273, -2.3563859108540752, -1.244465630559768, -1.2121696030437705, -1.108385229799972, -0.5997532535877523, -2.8417429456202465, -2.0174843747604574, 3.728418393324872, -2.4995817317946125, -2.1027364984159997, 1.204462171764829],
[-0.1295643139143188, -0.25180061016237076, 0.026840510348988048, 0.13838341357167042, -0.12405347333991444, 0.2013282899331681, -0.09716242657635013, -0.07280803791794475, -0.014863069992612526, -0.11865372078696443, -0.24336553886008766, -0.15302369060264107, 0.10656395110674288, -0.05621477319975135, -0.2423097874623608, -0.3559222655999977, 0.14223073270464587, -0.2619700169734253, -0.2838749817946618, -0.12770546626216808, -0.32874210418155375, -0.6080402579027839, -0.6013720311227521, -0.28219070637489957, -0.38574900522523703, -0.2854615595180963, -0.6638265429129936, -0.43243882008880186, -0.7087786525653952, -0.12262915566151401, -0.5621094360157723, -0.3724138415686157, -0.6407257262394876, -0.5778177668515952, -0.7266046689027658, -1.2406645463411237, -0.9524782619887271, -0.6746268533482955, -1.280321845783999, -1.1070170609005066, -0.3329616241429913, -1.783898796301015],
[0.11162850230497263, -0.05449108318477213, -0.12922902474650722, 0.08422075286457456, -0.22744180672149644, -0.007426259024856869, -0.09753103962690125, -0.08044050296016995, -0.02807570840451791, -0.35551601937832106, -0.08090172968843751, -0.07028401547088785, 0.04307560009712795, -0.14193233442966838, -0.142358290976249, -0.2012584494886126, -0.4526758891174422, -0.2174381500663088, -0.17532912059438727, 0.00430504479978592, -0.25830492751326195, -0.503619466526678, -0.3743153796690629, -0.5495332612513585, -0.2967792613271799, -0.3266537937545055, -0.4663728041700735, -0.3022435262812729, -0.8171176432481916, -0.41371595745446366, -0.4435264044826714, -0.3626288371701176, -0.327280459128235, -0.4303595047873503, -0.696226350566816, -2.7063159634866825, -2.2237074904987777, -2.725507108037872, -2.4978790069407606, -2.6038027940249253, -2.4669811243539908, -2.547735445515097],
[2.504825467542655, 3.023491892647893, -6.516706799293794, 1.5454100508070128, -0.5963897791733077, 5.05241544403807, 3.8011052532394634, 1.5358870275607956, 2.159729962158379, -2.554674824170125, -0.1269066524010994, -1.5281269270470104, -0.8589672885538933, 2.312986446318819, 0.8029074762517946, 1.7806577260952337, -1.0540505910930174, 0.08413781324486401, -0.5287196549628961, 1.7127819958436823, -1.1507048112248057, -1.394338117977701, -1.5694905758706852, -0.8559533654472561, 0.30198003057635775, -1.7130280955191064, -0.6165527022849858, -0.5279456113580896, -2.5498162006951817, -1.02814751166856, 3.23730114066587, -0.8982440921411645, -0.1988022053273346, -0.9414410031507958, -1.1548181490397091, -1.6347291432994147, -0.6450117006132604, -0.5349722529263835, -2.7520471173955063, -3.2505543158922188, -2.725226084709791, -2.8167323260588653],
[0.015419922931126984, -0.15544013727702655, 0.11279006456389523, -0.09250728287615666, 0.039169552892454496, 0.026600202615570213, 0.07774512868519907, -0.0647726510374398, -0.09509955845769605, -0.01595044162057733, -0.17310533682210394, 0.06511359764893872, -0.098001904028129, -0.05001218809867577, -0.23304940272832295, -0.32887536417343466, -0.07863544550839188, -0.23686693938640774, -0.22614763818398645, -0.21954920822402166, 0.05728173136499846, -0.036097689129852104, -0.16819960068272904, -0.28889594120831114, -0.5399330260771711, -0.2654353678057644, -0.08079548016695852, -0.17269040703214653, -0.6500012214338394, -0.3336460716643911, -0.5412259145262648, -0.29390472206143053, -0.4535906870639372, -0.40175966011515396, -0.3044216707868784, -1.183537413295506, -0.9321273625991636, -1.04238336633393, -1.835337147422614, -1.0809913382405374, -0.9851136056559762, -1.2415878722344842],
[0.770693672512661, -0.10401224898324386, 0.2714154594842774, 0.006486542301748897, -0.2575769129976731, -0.37541941972645343, 0.047643877141208595, 0.231027232783408, -0.07643677765855654, 0.34199193382059007, -0.07907432250433084, -0.34037974020175055, -0.3687609720497615, 0.5861937162913892, 0.560722183959698, -0.41586604786103876, 0.7529364752145488, -0.03387308168309807, -0.7364879791674745, -0.5955351236469415, -0.22452427634187233, -0.0004150275544944748, -0.3745992861261964, 0.6298534301985824, -0.9852956696561015, 0.8487104133121676, -1.3353458823177458, -0.10553837715154392, -0.759004540284062, -0.21084499868211795, -0.5527103484629836, -0.897946807366775, -0.16729525597864686, -1.4711764148585214, 0.4639026895704411, -1.6306416582022631, -1.7013698418271619, -0.0071242963621723455, -2.134916412462979, -1.6538744242587167, 0.3654516629826482, 0.5397886996832779],
[-0.08751524029562868, -0.3010810041882434, 0.4497326375015934, 0.7343446013109449, 0.43674283444090356, 0.34612781350222216, -0.3738354962015462, -0.11422019465202603, -0.32189699823364537, 0.26033748179873, -0.11358336541892472, 0.1433027734987571, 0.2684602319123297, -0.37220951049117934, -0.12127675278860756, -0.38844294518237155, 0.21634978882500694, 0.25239513301005934, 0.00018003205158074158, 0.1998946159738116, -0.2191631992521583, -0.3197285552920452, -0.03505675295151288, -0.09507987987678362, -0.10494880208680407, -0.581302740600808, 0.1644873849867466, 0.13038292644475338, -0.7960231756519697, -0.36350073521290127, -0.6091530789178572, -0.5087636815245408, -0.43907584877041145, -0.04885773973019426, -0.6311766450547672, -1.3001669479279625, -1.2821060098130077, -1.5142831843609494, -1.9206778702331182, -1.8252711482294424, -1.5464115778520557, -1.1242386626254726],
[-0.12676241531943927, -1.1191460705407192, -1.33728288971172, 0.6490213759171809, 1.9518898574994576, 0.9621541750044927, 0.43709013767604177, -0.24594739873835167, -0.48685347901687026, -0.9940997873808428, 0.07177662007213446, 0.6311573962342285, -0.060780913245674, 0.8432632583942097, -0.28012292685400236, -0.5516258974152084, 0.04029969913227122, 0.04061643223100416, 0.3485986979879081, -0.016814961728772903, -0.201119567249301, -1.1128974308603394, -0.9356856211413996, -0.534291170425841, -0.3749270718842345, -0.3846548689884324, -0.4960256930772719, -0.0092301028647348, 0.8394844761908515, -0.3916723120271596, 1.0484654160958162, -0.7936772537703982, -1.1096004383643832, -1.2793729419606046, -0.9035881521844644, -1.4285497321736007, -1.0424859387027463, -2.115298914252901, 0.6891229616186682, -3.435637919743872, -2.6132929102586537, -3.2060593980342436],
[-0.25079222240097676, 0.11897081732524822, -0.19648575817255165, -0.1405876074881539, 0.23917196854295575, 0.1406829755862927, 0.021399369246183474, -0.2912922797049103, -0.10757242157242407, -0.29715910106166793, -0.14762273493716915, -0.038043973200942764, -0.017811246160154764, -0.07881815004266593, -0.039722419663399545, -0.11166290824735722, -0.2341950991844951, -0.26483305043112804, -0.1699106986301444, -0.10019356191705077, -0.017299066818414557, -0.3141120451974891, -0.22780068555818103, -0.24888131904250455, -0.38518187280644556, -0.3035142838722223, -0.17705287051556934, -0.1916690379845137, -0.5618902396857175, -0.5551803029068172, -0.446999187058596, -0.08203255336201863, -0.4064334171524562, -0.6055295761790243, -0.7505981269943315, -1.9945412918718, -1.8941868224207081, -1.9803924560884505, -2.48469653957628, -2.4445766513669653, -1.9221233262685853, -2.378473149620148],
[0.17130853390816903, -0.060582788180205925, -0.19811539212963353, -0.3057409908114371, 0.02544576032060415, -0.07005962967691892, -0.22910413315894954, -0.15768076616375803, 0.027973285055699997, 0.0903604429904149, -0.3052042126623598, -0.030768448987846912, 0.01794043877113637, -0.1445335982121427, -0.15019156852716783, -0.24630592911405072, -0.12059872535654806, -0.32253808178965143, -0.24500990011699222, -0.30306398577246924, -0.10757824614328833, -0.3830401185059492, -0.2739609439290813, -0.30405595429151827, -0.4001441624200487, -0.20436309214188547, 0.031636509497434115, -0.11548386767697377, -0.8438324519163896, -0.46604438953589644, -0.660774988419981, -0.07790997770455023, -0.5493257789177451, -0.40460658480935086, -0.7578191901689455, -2.5337805726968554, -2.2456856026977667, -2.2808217654579934, -3.067498027391849, -2.684325790432583, -2.232936530061086, -2.5094822955820066],
[-0.5575264146288065, 0.8664206520195119, 2.165466120219601, -0.23543671439948785, -0.9745587457010101, -0.6539489648869895, 0.9410183530248967, -0.10803500159940937, -0.06935620884253249, 1.0901959040545164, -0.3602234511815126, -0.8896417903044085, -0.7413435570555073, 0.006049482750384318, -0.3576737349506474, -0.5001040953929661, 1.209465182689857, -0.8058593136195622, -0.6849630928249784, -0.15397466015725403, -0.11938299391097451, -0.01475186911514138, -0.952912921332502, 0.061004205134039084, -1.010697361529318, -0.7225506371694901, -0.11140288584807834, 0.3912663175452522, -1.0001395269975792, -1.9197109449937615, -0.6735328886287086, 0.5343350220879917, -0.18329820113455506, -0.5549932358339819, -0.7775133251744154, -3.2238894254214765, -2.949443844904154, -2.981821679361744, -3.9172613331107593, -2.642220166016715, -2.7496521755352625, -2.7809246176084614],
[-0.21881233403939832, 0.2115221612120584, -0.00767514450002479, 0.02924071631769136, 0.3158806882499726, 0.22160290545738082, 0.10652502371733956, -0.22466993052099352, -0.1611960383160778, -0.3141872875975008, -0.0163168244467225, -0.06437054887135578, 0.11878773948607764, 0.1107915244930391, -0.4106193761472203, 0.23844995820589948, -0.3321523302543906, -0.1544704972492812, -0.13473553389559206, -0.28029695440887903, -0.028045898888308704, -0.1876140537741546, 0.03736619512654668, -0.5103514706559316, -0.3548082483811509, -0.41988596638403825, -0.2641918907898408, -0.24099026704378265, -0.9669637206544891, -0.18417593396552948, -0.6529457861048165, -0.3247584181670291, -0.5160502738350409, -0.6740018491063906, -0.8590028433386125, -1.4172622433504627, -1.731481028821185, -2.29628972177208, -1.881963925566943, -2.5325525749751057, -1.7534641606526082, -2.307958330856875],
[0.06542491203867923, -0.6576721945557111, 0.8741125389267648, 0.48902058029920337, -0.09460321384946521, 0.5323767030633743, 0.14004244754393508, 0.17543345690497267, -0.32270203315442325, 0.5188752542108422, -0.06038430564473857, -0.06463495872293742, 0.3207877576173293, 0.17443878086969114, 0.29212909509659657, -0.06101878536409165, 0.1746747116411132, -0.3140665770682368, -0.023984566712023415, -0.11110413554403013, 0.007322957090619532, 0.10230894576473173, -0.22345264396471293, -0.15542750360224464, -0.12902858842927906, -0.13287012404727708, -0.4900827048598215, -0.3039898350835402, -0.4724518880379046, -0.1745096583616689, -0.5171056187410141, -0.1656731566773112, -0.17120825850502075, -0.3647657780404667, -0.5313190270842433, -1.2444713958811648, -0.6865148152326951, -1.2892761600222828, -1.7290776632699598, -0.8816974465249692, -0.968636538501159, -0.7254265450141628],
[0.3106868616836474, -0.196415859622959, -0.6267374494983798, 0.03202373314476346, -0.31954520924725277, 0.688893396101471, -0.25478652787332723, 0.2170697375632487, -0.14613580109018073, -0.279678274302803, -0.40299408805906767, -0.06309353600522806, 0.4251846867668318, -0.2817527053993157, 0.01169784978020753, -0.21116142545752584, -0.2551630009683355, -0.06025082227976725, -0.38023944933917797, 0.2898684886618371, -0.2946861254845581, -0.19427707802558875, -0.3853524708139743, -0.5564617144643691, -0.00506043474337324, -0.49023101039708217, 0.010690668916258902, -0.05471831419292206, -0.3591925029795019, -0.4130282368102847, 0.2569728532235989, -0.5070785788820785, -0.4621661295186348, -0.7410915023000489, -0.5302261586362406, -1.8091687279912023, -1.3959784176445453, -1.1255889701745712, -1.3900894207459604, -1.8436924397441812, -1.8085190165840366, -0.9716933520940154],
[0.2702458177173488, -0.6301607730600469, 0.4489555669997414, 0.6754683371894057, 0.29468079153456245, 0.3034318666050687, 0.007046393855351821, 0.38645683808366493, -0.4906988537084066, 0.19228794290316595, 0.12776030120363846, 0.36202723803153186, 0.16594613329732205, 0.2572157153610317, 0.29381229391331193, -0.029598404042426116, 0.0686238742947629, 0.21178329545347915, -0.0777525640801308, 0.14920335002241017, -0.10974549934740846, 0.014487370718749074, -0.47763043753813744, -0.04693829442987868, -0.4726855461925921, -0.22368005462642124, -0.18865921195483398, -0.31036222536844077, -0.5647534017267958, 0.024073049987195803, -0.49380887959319736, -0.5228669916034503, -0.2927307822181733, -0.22316412304101416, -0.5348127716352693, -1.2403408303325252, -0.936486246194048, -1.221595422411258, -1.417609380368732, -1.560229443519466, -1.2364345337477545, -1.3210504160663754],
[-2.7865128106657036, 2.9174841816366017, -0.04642190979595713, 1.71693553787226, -2.596438437082425, -1.9290876352337514, 7.288904428964901, -1.3382253593056208, 0.7580261553310939, -1.2556181238475583, 2.0408798062974154, -1.5157247103356444, -1.0157823012695457, 2.9237181296452723, -3.119658695735249, 1.4041082303197228, -0.44433190580541776, 0.049412759901043296, -0.4837909142565302, -0.019001866897903, 1.7055743956347362, 1.5101233682419857, 0.3854065585618159, 0.28109510360823936, -0.0979250905553407, -0.5419750764279349, -1.6142919930030575, -1.0990773967885885, 4.600988469561021, 0.1503413613925975, -0.17320860398572596, -0.7406799877640228, -1.314902442448378, -0.272922674278732, -1.181885140718168, 0.40598485660083783, -0.7283131139157609, -0.1202503280278982, -2.968440776906975, 0.9931967911545803, -1.3607024380523332, -2.5578702241452547],
[-0.19665405333413422, -0.31980542107619864, 0.1490955071589639, -0.5795075220488922, -0.32698504069531853, 0.296110934069452, -0.20396388636158713, -0.19577013408290297, -0.0032971885173064906, 0.05686747276657215, -0.9012628835341538, -0.154117712136529, 0.8403988521442045, 0.13554239839838442, -0.17531118042113558, -0.009179237317074403, -0.19283556274476135, -0.10603000752077216, -0.3578976199725433, 0.4890710819494253, -1.3350969756964997, -0.25832241079120416, 0.1989139513954744, -0.6750985300684199, 0.9396320910218368, -1.1807605125645575, 0.6538154779318057, 1.1199488160771798, -1.1332852314216058, 0.004132797118390316, -1.2978549344611177, 0.5043347813223645, -1.5605486340311412, -0.21683772968061063, 1.0860906222072493, -1.4953161686949434, -0.9970411645272331, -1.7103862804580712, -1.7161824747935348, -1.7716295409726515, -1.8740991660024295, 0.7663753112805225],
[0.29549323278400375, 0.01171531187195369, 0.37457617122539283, -0.23389619871237, -0.006790877630933169, -0.09223522073239629, 0.06181994585265666, 0.013420381422057422, -0.4480456566090479, 0.06221999561687576, -0.22748410532800573, -0.05115501748386531, -0.1769703977328226, -0.25656780572114374, 0.1718882989308781, -0.33736363493716026, -0.10972883173868211, -0.2851288736777983, -0.07145848086599238, -0.21405667033363318, -0.10132865957551822, -0.1854935279773466, -0.5646977081096388, -0.43272628858448325, -0.30515228969410213, -0.5516016295510147, -0.31681468595313345, -0.2548656806112417, -0.8621189858040466, -0.46004722778424245, -0.6807883400890777, -0.22514521398943038, -0.3515571837251321, -0.3350938046979895, -0.3527027368341997, -2.127626161188563, -1.628255283958867, -1.5842676394160438, -2.1150733043921814, -1.5887922461352344, -1.585608746783932, -1.5505228183369895],
[-0.14655844587394853, 0.24458487522760525, -0.0769299405471108, -0.25561470919397977, -0.10256594436476156, 0.024385544561567336, -0.2107965888917489, -0.22598116457189216, 0.05012181258388202, -0.0636600479581005, -0.17527117815402493, -0.3381951467677002, -0.01762446909113148, 0.031027685067476676, -0.03326473322924235, -0.11220469429239578, -0.08719124895792339, -0.1759579936017748, -0.18173608822920867, 0.04229358083118558, -0.13540588530153813, -0.18412699243642047, -0.26413088142817603, -0.11626318398089559, -0.427240350601403, -0.3676437061220718, -0.18406072238302743, -0.40537348267942297, -0.7878887113712988, -0.5530407714332448, -0.6020803598388217, -0.3519015012560876, -0.322663806548191, -0.7320225697107142, -0.7069747993222927, -2.2372005642255375, -1.97597966835698, -2.4624216703094297, -2.807436575897918, -2.474081067688865, -2.1753411658880317, -2.0679846896741747],
[0.7169394194411874, 0.13923881815295905, -1.2141904108527093, -1.0360502539126564, -0.35116983458752715, 0.34872827493669994, -0.20948608392817972, -0.2981072050250316, 0.004196117322046308, -0.7913677871463012, -0.16529239151890165, -0.533016068640789, -0.033903506832719976, -0.5724260805630864, 0.2909907966659764, -0.8886171034936079, 0.37550561074342537, -1.1072183532204078, -1.3943612516228512, -0.6112133246609206, -0.9172888395741928, -0.027417106502452132, -1.5808214508401972, -1.024883643936716, 0.6139384723148541, -1.9956572513945354, -1.3933801030462478, -0.40798138684835833, -0.8456491071919383, -1.8081829931682785, 0.044461760734265596, 0.3036230648853919, -1.650173160277084, -2.218761816269357, -1.237536011515815, -3.167925887300578, -2.822213753671106, -3.4028492861972817, -3.0747073553045747, -3.2176958602543024, -3.398055562698001, -2.7631815840418916],
[0.5160452814099861, -0.042468740390556196, -0.004173774029409005, -0.04512172563137592, 0.48433516843915964, -0.3320028182358733, -0.15339559806347364, 0.20349950866096828, -0.4330228502968814, -0.10273920436182869, 0.2770359887804321, 0.04253316334380667, -0.2735214419225651, -0.34946218331784545, 0.10785475463719835, -0.31073902577402907, 0.04751170923070697, 0.06522158610798891, -0.1331461233725706, 0.05863619675184083, 0.23211808543970588, -0.17582124193957413, -0.2826954253117365, -0.09445901344449133, -0.49116637000524177, -0.5823261500817316, -0.3288940355867549, -0.2360984524406449, -0.5453095612860236, -0.08850904350802745, -0.6745544944716402, -0.5014555815759647, -0.9530161838238029, -0.32855454463808187, -0.26728051817343984, -2.0563537686492284, -1.599606935648906, -1.700309448005109, -2.0871518760780967, -1.9205986358477238, -1.3583097738300631, -2.0774521764520295],
[0.2696543409801525, -0.1103037406713191, -0.04414703262228328, 0.7612156114942918, 0.659006575158394, 0.6633601177408709, -0.7380404295335274, -0.03359228935734337, -0.26022311774628437, 0.15493847155254237, 0.5140176328118203, 0.24017550300646107, 0.4483502223319963, -0.4759024005899674, -0.14224528451472418, -0.20970439266438792, -0.10368537765636743, 0.268643258520273, 0.023812146538806894, 0.04977283841509387, -0.29432591433377897, -0.01779601535757367, -0.2483742562325502, -0.12262077042886495, -0.007824821992426883, -0.28034291070515877, 0.4105339728154395, -0.30552798079616517, -0.5409323761462902, -0.19469339211700942, -0.6223406253893805, -0.7559456641627177, -0.49687831972568397, 0.20232743168013784, -0.7529564475734198, -2.2867072753482414, -1.6977135256191573, -2.357301503524571, -2.5549356680859607, -2.3705533220091635, -1.7523881067698095, -2.0127238693165386],
[0.025170935422676906, -0.12776391344826485, 0.04015892884701663, -0.1937204475461952, 0.0908987680539061, -0.10325831114089241, -0.17983906337607902, -0.0010433527752716982, -0.03232530734430222, 0.039237006108162845, -0.40496094236810587, -0.0452045524611411, -0.04155154626362935, 0.08394902487078572, -0.23859640202647175, -0.20229125366838088, 0.09722727241489577, -0.12392439064910679, -0.1543899401955538, -0.32959062418233703, 0.06785081978448768, -0.21534434543924338, -0.2285259304066706, -0.13717018226019756, -0.23431815091048402, -0.2180473553953417, -0.4005260005353066, -0.15858996481895501, -0.8853627576134236, -0.4278228985025149, -0.5626430021598663, -0.14803154584163047, -0.5249971916681913, -0.41704518344380337, -0.9249249940666636, -3.3757236968639965, -3.000404127483582, -3.327473375273509, -3.4666035825427692, -3.389402280344644, -3.1534500260380547, -3.330155358611297],
[0.012334132985288754, -0.2328336504610504, 0.19557733226636528, 0.1919575076378125, 0.02868440620788767, 0.009836390088012665, -0.012937966700938148, 0.03708003037029625, -0.21002699182654824, 0.1293549102007254, 0.06034364538903509, 0.10010832707415583, -0.06024246712295103, -0.08076670418532593, -0.14734334160402862, -0.14118268936440337, -0.25476508702784645, -0.04875744999930335, -0.059673125194420404, -0.05095100882287223, -0.16480925403797247, -0.44185554484896866, -0.03019871438324581, -0.23536472575852013, -0.1856959923260207, -0.5119424487107068, -0.3547191665650716, -0.43772248301779554, -0.8007405867403614, 0.3598654120438344, -0.591713501117622, -0.4529321023109895, -0.3735882046541311, -0.1880081150175929, -0.2594594500324528, -0.642537090805131, -0.8606674649855157, -1.4209674525374743, -1.5984294628467746, -1.4097057264239639, -1.391234533173599, -1.0523735427708873],
[-0.03230976626248373, 0.04740918967600583, 0.08653322559041987, -0.10294503437796075, 0.034946200541301195, 0.0005302754307818399, 0.34145240765637225, 0.057854851786309905, 0.09152063167110268, 0.09094234164849135, 0.0213600500834685, 0.18738106562513862, -0.11480522384592702, 0.21776145985561973, 0.2790826988534948, 0.42225192179857735, 0.24778568478375762, -0.2508359366401294, 0.4637945313501675, 0.22920896213244923, 0.09775067557163919, 0.2818357152980592, 0.036637552367956155, 0.2517268877919037, 0.16609494986493809, 0.28568820532113626, -0.14162161198756906, -0.2531770022401883, 0.4050667736026931, 0.12479777978565829, 0.514037672531018, 0.21044577314773966, 0.7104445687065616, -0.19230428181725479, -0.2690778616707053, -0.21604479483769237, 0.5400032593126507, -0.16231961752889204, 0.4971154689357309, 0.40211554051195375, -0.1880061495249472, -1.1832669808521494],
[-0.10818481971780705, -0.011502865869374903, 0.2575614152617602, -0.1240288872635478, -0.04516197520320013, -0.001333929553777463, 0.08747852211533387, -0.07930528873230012, -0.05995885578490058, -0.01584758189329047, -0.23061862192488425, -0.2093450000162061, -0.07552151378743996, 0.08786434095096199, -0.042266037887003595, -0.1261843797695658, -0.04399681830696516, -0.22676727727664484, -0.006402986442341262, -0.08221725739252632, -0.17934702473928757, -0.18772754321745994, -0.24771601668575655, -0.043867013453174036, -0.3826839388966425, -0.28857825467873444, -0.3216359334680838, -0.2014591145489557, -0.8854932250621661, -0.4480099911129872, -0.5161756917578535, -0.36760736367307295, -0.4382854614069151, -0.6096207467909246, -0.8372365078012222, -2.3945083900935256, -2.1133100168217713, -2.484070537131572, -2.8797813711809073, -2.530882322812929, -2.2960682241620844, -2.4502970504809416],
[1.2361050792366473, 0.44530286137218583, -0.35363047682431453, 1.1627817438523, -0.1965115348826878, -0.40689005686545365, -0.22608787541828054, 0.7459293863012735, -0.05379470370608561, -0.23163747959418363, 0.3037369739337864, -0.29997053040939237, -0.5870925091539844, -0.4314499537684046, 0.04017244214927485, -0.1510197415975227, -0.24532416048205286, 0.3312948241311646, -0.2008282402175759, -0.13490327768382848, -0.3038227651437167, -0.6731098349462787, -0.5367581807930448, -0.06460327378247081, -0.1316181915669515, -0.7055408063071821, -0.42366185419804525, -0.3488526597787998, -0.912887286718892, -0.5297232008861225, -0.4318740267346569, -0.6604015123149298, -0.11487421194994869, -0.22855022752214663, -0.7480160139864555, -3.1409386449881866, -2.614538043885291, -2.9803123055863345, -2.5345049000531596, -2.9520086728493338, -2.7326448620138324, -2.693275563047242],
[-0.02321502141249586, 0.07431376715522074, -0.31520083604405597, 0.11201098777625125, 0.44033190909445247, 0.3152728891173755, -0.17113620589288378, -0.11224311417945713, 0.039807447326327385, -0.01616712828901257, 0.11797204348084095, 0.1634613915873031, 0.3294518153713343, 0.053147188677673415, 0.11885040993747803, 0.1475200281984121, 0.4527080757718222, 0.039022752391883936, 0.2644007076988218, 0.17428177391313418, 0.08036605072957395, 0.1947199529551566, -0.04856936178653841, 0.6673285680268085, 0.32207819287105377, 0.2733828420030193, -0.07055268598848942, -0.26201789126567315, 0.23962332109045215, 0.3823868080122547, 0.8430926296783238, 0.3247475509225306, -0.10064615061041754, -0.21678589861672634, -0.36472798077850593, 0.22626944038245556, -0.15380314365354888, 0.9321723485754906, -0.9497231012079658, -0.5656763445045416, -0.34567350430383176, -0.6886674613786448],
[0.5892050019932744, -0.003914090114298237, -0.06501427779968134, 1.096443462675169, -0.6606973696163547, -0.7854652179713306, 3.32265158138615, -0.015990752422052723, -0.5068664983477176, -1.6262380496743392, 0.6415075647686121, -0.6859332238722793, 0.09391372991695614, 2.3348193354243723, -0.22625610054637155, 0.021488453362338265, -0.4457849519801288, -0.32369051570663504, -1.0000769356896426, 1.0007053867399078, 0.9282907804812037, -1.0702931833680789, -1.1067215935662744, -1.062696943964411, -1.2412576003043956, 0.3605823517750038, -0.1275298630504919, -0.5405689500497184, -0.444047470200217, -0.5151925788486843, -0.12240249123076394, -1.230197326932445, 0.4518646966293991, -0.88195658188655, -0.9995349731062337, -1.4040855115131454, -1.827002147295345, -2.0072637015181556, -1.4225045696961423, -0.6855740980053688, -1.8678792526077759, -2.5824458649390776],
[0.07848412394013936, -0.002110621147882173, -0.13415669928361618, -0.10896053225802958, 0.06706438881790278, 0.05657312410608547, -0.06906599323807423, 0.014770376834125121, -0.2523077310658839, -0.15788624795833303, -0.18189924286977757, -0.19505176748307282, -0.2766400783223931, -0.0010228473191932319, -0.02276595776704428, -0.19491784300044301, -0.22686573833433785, -0.39492043123951803, -0.150858958042565, -0.3564418252352274, -0.19272852763687623, -0.3782096917447158, -0.22494157350219843, -0.3324144068043275, -0.3836735117588065, -0.05643331356961531, -0.34128014391490236, -0.3411890038164858, -0.6153253575378509, -0.5167069133075509, -0.43660830806135836, -0.32063440992815406, -0.46092734187399187, -0.3935742254156609, -0.7275351067455444, -1.4514513624069256, -1.2525722589703816, -1.5014142508844797, -1.772966968571951, -1.6593385439997843, -1.478915051898443, -1.9119015715350407],
[0.5921509656286736, -0.16170928737305537, 1.5422482048802386, 0.4500408362997015, -2.0702231830521267, -1.426381150776971, 3.227555915706385, 0.6078623285527107, 0.32158029822529205, 0.7903964830027109, -0.35976494485424515, -1.383555546019524, -0.5367432165006915, 2.17186244725904, 0.08745792583172642, 0.6586834624598468, 0.3443215471244312, -0.9470574476184707, -1.353514878403267, -0.09360224431462091, 0.3049895731145022, -0.31684263292242304, -0.4805534692127991, -0.9749791381023857, -0.47725339906729747, -0.7968894040270923, -0.8721085822779742, -0.33101683465978, -0.5520623735864649, -0.8102208313577096, -1.186384949314596, -0.6545472732144759, 1.0790561806223549, -1.1046070551090013, -0.014567359389011143, -2.4386538347647018, -2.065070764302698, -2.228525496328283, -2.8177375099058986, -1.1611524205366988, -1.286144159307703, -0.4245128145748147],
[-0.24824075941775417, -0.7321113376563161, 1.5829071599139177, 2.330771305192028, -3.756529730796256, 6.231050077103939, 0.021515866494493385, 0.33938450124899866, -2.1289748921170872, -0.3479635034392541, 1.0804940430503092, 0.3992849348658551, 1.7950747453869642, 1.3638040825709834, -1.5895481740152002, -1.0214605213409975, 0.7068444024789935, 2.128540689415948, 0.29549850377177533, -0.41283390542014836, -0.963836769294475, 0.8089957553578652, -0.5074851441362621, -0.10697003904348171, 0.3918471975593875, -0.32462745682281163, 0.8955196139580812, 2.5105006917356505, 0.10763759827256589, -1.7934138390231278, 0.7469270450187017, 0.01627128789348396, 2.3828634983558388, -3.921116720304023, -0.38840340804862256, 0.04034142346658496, -2.461924300477805, -2.534635044604597, -0.7294331455314008, -1.8559642933279492, -4.978474790265767, -1.7884586626790984],
[0.21098982152894274, -0.26392109840320604, -0.5000398676975732, -0.34709573682641537, -0.06937158089403732, 0.14431783201253007, 0.05426171596716834, 0.0777722108781787, -0.2662425591371217, -0.34940688647862517, -0.38052433778946876, -0.1344922346124856, 0.09022829318140689, -0.096830376485605, -0.24166271275446025, -0.17816194721572676, -0.4240603778447199, -0.3197772972330699, -0.006353121117243876, -0.25626148855763276, -0.3545904438897095, -0.19714340989060358, -0.33270110011992643, -0.48176837659208016, -0.3183236861374576, 0.11083613613936232, -0.17016325724628192, -0.36807881010596516, -0.7042594521370692, -0.5947980527689392, -0.17843595335577572, -0.3807394236150423, -0.30275744940450994, -1.0147145986033443, -0.8311598210686602, -2.474692892308697, -2.167572155097639, -2.3283932326145496, -1.9908231940128431, -2.774375031440262, -2.461047198957322, -2.978227138970594],
[0.09723961304991083, -0.07701111051211681, 0.08643433917500133, 0.1982813894074231, 0.2504557295645061, 0.08093373671709354, 0.03051038780297766, 0.13170832992653378, 0.024589777967802477, 0.022852882095405078, -0.14114811399325955, 0.07947054330695731, -0.004871986614070024, 0.06229661637176089, 0.1682900300709809, -0.05421902086325039, -0.33536816911021267, -0.19978762010817744, 0.10644923164124735, -0.10818090722013507, -0.05490434944603286, -0.3610530746917873, -0.4494208021079361, -0.19686860295236042, -0.29592833751411185, -0.24903783438606614, -0.2372919126631211, -0.38477276690381096, -0.6223587435301203, -0.1343967243940047, -0.6168910912974488, -0.608870200322844, -0.2384376305286786, -0.04336985341592366, -0.4728721516243064, -1.9812754412723945, -1.4077094629772784, -1.6750538999557347, -2.136297684112554, -1.5301330324379843, -1.531812274657718, -1.7216281436939107],
[0.16288509592483885, -0.6048470829827962, -0.6375553291055122, -2.525991648133727, -0.2764419939448663, 1.34089645929428, 0.2009278637120778, 0.6763793967595213, -1.2593012510511588, -0.6360276209179976, -3.556533444366871, 0.8841846453016478, 2.2748916770888763, 0.46220972965380475, 0.8813994655991068, 1.6698595600348405, 0.3015938352503988, -2.7389449558551533, -0.29393173730682787, 0.9694311129405474, -0.6830149249839561, 1.1058417556173552, 2.5692362874188284, -1.9973682389174128, -0.608793129759564, -1.9496945976842364, -0.5420807014437314, 0.12868068042292888, -0.04067146954377024, 1.6861223815336244, -1.7981834988598009, 3.3133386680728134, -0.8651681033756301, -1.6632825906142028, -0.6281287212559186, -1.0970055958653617, -0.37800822012400964, -3.2878939094016406, -0.9916517768426615, -1.111793713826146, -1.1943628633389078, -1.0998090227622936],
[-0.12938744802978197, 0.004825326398847968, -0.05953859677304469, -0.060815856427248995, -0.04851156699827646, -0.11027464000715623, 0.002135326306234289, -0.17104484685191945, -0.20987836950949104, -0.2497526495129107, -0.2789890217950906, -0.16360079636328123, -0.015267355454183049, -0.1602295041231459, -0.21925839500072442, -0.1879611530452783, -0.30351293205939683, 0.04869676747849898, 0.008749232146409795, -0.21328420719040728, -0.224340775109703, -0.1263230241099447, -0.5099146497974306, -0.23576335397570142, -0.05349878342540369, -0.36546268561046463, -0.4679004467897562, -0.11345718517506995, 0.08700379148155571, -0.4267797183348937, -0.0025847494108277146, -0.15904320146101203, -0.15864459964656394, -0.48341228767960726, -0.34997150819420003, -0.9109643697196497, -0.43113874615818715, -0.14599823287404112, 0.08715718241585999, -0.5654075769097927, -0.5995488474216927, -0.984002484483477],
[-0.11735357922883616, 0.0383030380069932, -0.07649775587049071, -0.20462615182872215, 0.0005909583958362097, 0.2746689206341783, -0.027426454430890573, 0.0538577729775755, -0.27392515061768785, -0.35979620870127105, -0.2944373623061038, -0.13260474373681277, 0.08517733612261262, 0.04846926938563818, -0.3271505018812343, -0.1807003280221878, -0.3686555118428069, -0.5486474022313849, -0.12232549810643915, -0.18172995799054442, -0.12950917912002663, -0.08415042111437701, 0.0018831896192685839, -0.5225044874933494, -0.42646224339216243, -0.5796248071691616, -0.36777367252266674, -0.2637747339750696, -0.5900175555290135, -0.7153105379410414, -0.6615840527837736, -0.18445858073970264, -0.5863434132167807, -0.8385877302929188, -0.7897929634052508, -2.8860059310148327, -2.4902792257918946, -2.6947556964184596, -2.9602297303611196, -2.8900489674499577, -2.5299747002570316, -2.7059204685887797],
[0.009165285967854871, -0.14662917066644726, -0.02669476997333429, -0.23663900767756754, 0.48736694606954095, -0.20552450189839427, -0.04799622165851068, -0.15784674957969133, -0.13665828612967656, -0.040568780925439155, -0.3208939673279612, 0.15383131236400935, -0.19202100669080707, -0.10714418810971249, -0.11533224706246994, -0.21711421465797195, -0.27071730512271813, -0.24843066574020475, 0.07204667402980387, -0.06601725286947654, 0.017972959145787462, -0.11839868488412549, -0.29858749796754547, -0.3779032175590407, -0.37289573588982716, -0.022151888983640528, -0.18053594241611, -0.29533958800195553, -0.49160331011136166, -0.34257314677198525, -0.32953557089713975, -0.3974592084285706, -0.4835750636195353, -0.45493648190100977, -0.4382091206629727, -1.1697360406820014, -1.0335596446741335, -1.0591893517703392, -1.38629651225298, -0.9775575338503709, -0.9350418990559298, -1.6351391579146344],
[-0.6879199213937995, 0.8099863044964184, 0.9445999042463991, 0.2974772186492334, 0.9303431416553498, 0.867819792639027, -0.10680460490857258, -0.18973480974608353, 0.4524219374442999, 0.5075886932514884, 0.08889687108208624, 0.2597947110208893, 0.2774173356722446, -0.023797523608582206, 0.13262013769345377, 0.26407923421870005, -0.11434046855590725, -0.2180359847666116, 0.32489062584344686, 0.0600471428613073, -0.004793519699001884, -0.1616663675924113, 0.529526867677655, -0.025373691573353153, -0.3440476561239314, -0.2933447746309606, -0.28912852741118233, -0.03077652348345338, -0.9963442197046435, 0.5872416502897598, -1.1369467654844085, -0.8472547247867122, -0.4901030517649658, -0.3357185838272285, -0.7050975693912992, -2.0631671445604454, -1.684830181024174, -2.726741889826106, -2.869910179184111, -2.5717207763804084, -2.4012900349295263, -2.3269933650764427],
[-0.024214569374960394, -0.2947300838375349, -0.0038905919139751634, -0.17136654332060142, 0.05191606876804656, -0.14603161104864462, -0.07176742449436682, -0.14146112214713658, -0.13994315151837333, 0.050564098709308614, -0.25434827019762474, -0.12915632890248793, -0.12797202912854075, -0.2387419433428378, -0.23202656392130372, -0.3611320370899064, 0.0310663921548957, -0.25169234453305545, -0.049522312997033927, -0.2256000028177155, -0.28457297802141107, -0.2387453709156573, -0.27605318318748895, -0.1652542693102384, -0.6067317479113709, -0.2509319077692336, -0.27822550315744743, -0.470103534241803, -0.6848560441594522, -0.3950189604118649, -0.5006650461495876, -0.3104050097783844, -0.4363008080850815, -0.44422573875482324, -0.641215945064803, -1.8017559048814134, -1.3746849252512747, -1.573694189600858, -1.9645129392788998, -1.6222361973777097, -1.482218365334876, -1.8492634316651293],
[-0.13405643425260652, -0.1514969635617994, -0.09326582588819735, -0.1309386713857912, 0.02200317681691004, -0.06390531175186465, -0.07497417029137868, -0.13002848850293633, -0.03832136589493423, -0.18358086733167278, -0.028986055628670144, -0.2206245222702739, -0.013942348310246917, -0.10862104635492766, -0.2683713649459242, -0.19415876243617908, -0.2516083894846842, -0.33275810396970545, -0.43464914382475556, -0.08939196842223009, 0.21756432016957664, -0.1821485264126925, -0.40322388521964153, -0.38243709169260726, -0.35621875531067787, -0.33018780650784885, -0.19745956690508984, -0.06700851870387374, -0.7834382660548294, -0.3266661738721384, -0.5354699876269124, -0.34938592172859373, -0.5823326484620666, -0.1622824681700076, -0.6904359473592316, -1.8287989272413696, -1.4719542378906285, -1.8333841541858655, -1.843534061942341, -1.6864261835078593, -1.397021859892559, -1.729869801995165],
[0.21888893183484368, -0.003251370883993492, 0.07929156353165498, 0.33216574337626825, 0.0917845604597854, -0.180759721945945, -0.2533037401537404, 0.1181946957704316, 0.022585424988380207, 0.07980961297534456, 0.05516188026632965, -0.17147059283598176, -0.22184086937255942, -0.12174823088660253, -0.10569054375010521, 0.15677605245174206, -0.006454492370741542, 0.019008422101890837, -0.11369988000640086, -0.11579823770494095, -0.20276995009072435, -0.531022898720646, -0.5330923195413481, -0.10352687707593226, -0.09224525906485836, -0.5271206923692161, -0.5220345677317564, -0.15259081975445327, -0.599274866162987, -0.4226387205320665, -0.4120708561568477, -0.588839211374412, -0.6261647772078393, 0.015843495217767206, -0.2646721492899315, -1.9084689126380878, -1.4001790350441352, -1.402458122216957, -1.5306931352007886, -1.2919354303398967, -1.0807999364015286, -1.1029939769117196],
[0.8674700773818164, -0.06825049041051287, 0.20066701067718318, -0.5340499082647906, 0.04530398843868153, 0.23974666880010775, -0.46932590668613444, 0.22131034285249584, 0.17316727605879814, -0.44792394273766023, 0.5332251326174923, 0.2860402186432821, 0.23892495927491683, -0.5499714458363913, 0.354312666842072, 1.403699012315916, -0.6637357755016576, -1.5610879358824203, -0.38861035855250636, -0.5997953259541239, -0.648122647443284, 0.22646238150520814, 0.7480982931789606, -0.14010708388103535, -0.38014186335505906, -1.0575976632247837, -0.371296656004181, 0.11959898698248247, -0.3788252683439844, -0.19829016252273546, -1.2848552011556724, 0.6626423804008894, -1.1877025298808186, -0.4746404573184427, -0.2349138012904488, -1.6770124394301378, -1.0361858272038822, -3.229586737773154, -1.9713679607001595, -1.2792786184592548, -1.4555166027623103, -1.792475228342507],
[0.04053617289957361, -0.41726768578770024, 0.06828704918827712, -0.3277327262201459, -0.21781058776071252, 0.2061684582527765, 0.15650995722070346, 0.01642091317831315, -0.4338004708476368, 0.16531123457475433, -0.5092722147932435, -0.047801743745398, 0.033986580972802674, 0.2113037337856126, -0.049959369553104906, -0.579003518510821, -0.0629815488683231, -0.4196528886559566, 0.17870640923825912, -0.19017332507310847, -0.07817957360114239, -0.3851573487580332, -0.1119597525127342, 0.20654791821832838, -0.5038659904144703, 0.4325071297231642, -0.09201553152114221, 0.1849045013277176, -0.5714519829157846, -0.7289785543043126, 0.0785713213456424, -0.20182891856536847, 0.23741221861688766, -0.3819729592463902, 0.8982705912386612, -0.8887890161493924, 0.18316195553949405, -0.313828985391215, -1.660222197388773, -0.08527710756314355, -0.35869434980825465, -0.1788023869105718],
[-0.15565314771576536, 0.07085322993345623, -0.00019881409784206974, 0.006418403569775105, -0.1500261585568077, -0.0534198437514119, -0.01561043899968022, -0.10633003717133743, -0.05634352807971404, -0.41779176562509646, -0.3061885618017398, -0.17131128395394965, -0.1155382209722844, -0.18717745179241296, -0.11393586127545721, -0.3055758598964943, -0.21677862632362926, -0.3182325033061489, -0.2790997000910325, -0.3972196044333284, -0.13731746895650165, -0.10485653248018541, -0.13767638214961345, -0.6605586136034295, -0.5494649004156369, 0.03512683103107068, -0.20070467069583003, -0.2486517456850728, -0.187999698939483, -0.6894488293548588, -0.08328352377479696, 0.2173314286054916, -0.43516799404809337, -0.6066871098886831, -0.6345592290003226, -1.2078380810257752, -0.8551908088936966, -0.8128406703634353, -1.4684301911139608, -0.9464277406525775, -0.5979532070910663, -1.5021759213092318],
[-0.4442636392058883, 0.048718637649822316, 0.15738080666780424, -0.08340905620649562, 0.30260543184291816, 0.18255254630556986, 0.17877038296815015, -0.3191660673933714, 0.05369606044612964, -0.3143829662622905, -0.08899348888235996, 0.04803337913640883, -0.031551084637702155, 0.10441201480610525, 0.021735351683697803, 0.008507990618958575, -0.1611622560684709, 0.08459421606506093, 0.05954169803840434, -0.15898017325090114, -0.05514904325550634, -0.16631244957097763, 0.03347792452709337, -0.131135024527517, -0.28287347426456516, -0.13420918699841777, -0.5217295413053894, -0.21179230380126252, -0.8028697539365119, -0.34091168320356624, -0.1865483360606712, -0.2032766120284095, -0.5048551473535422, -0.45321322020881366, -0.8590429516422572, -3.322753301436267, -3.1461970185132158, -3.5682358440573405, -3.3481443562289455, -3.613701128160455, -3.3204729313864974, -3.661796962720846],
[0.048993648657083104, 0.17601366515842598, -0.6118337340082064, -0.3779372981137498, 0.4140795353208356, 0.41321052174892137, -0.15030241999850102, -0.0832306469303961, -0.054319559115614834, -0.730588769119601, -0.3210352598850052, 0.23383186214722732, 0.20915905897099102, 0.07650492537735516, 0.07873142776418317, -0.15518492353112212, -0.40284871772781133, -0.29078953786870537, 0.2779883121714837, -0.20265528799913535, -0.20869770548395697, -0.23225798517805224, 0.31482552029844824, -0.35773473381845367, -0.5982764764968439, -0.032976080694491705, -0.5641064473286724, -0.21634785392892475, 0.6049677480911729, -0.12558547010890272, 0.1671589711205656, -0.4163262287508568, -0.4711827670750883, -0.8013959151702156, -0.8217502818696574, -0.5639319739560151, -1.468267510734569, -2.0631326406656165, -0.6076072882931397, -2.2452299823043003, -1.4862578884952589, -2.3639213732400473],
[-0.12942616337178137, 0.28968688255524344, 0.0810092046046525, 0.0432080352030919, -0.061530367852081186, -0.07905747801306076, -0.05730908603880328, -0.169077742672984, 0.14525323843476562, 0.10047108020715642, 0.10836836217187686, 0.02183050142837334, -0.05720687638600162, -0.2931817258309844, -0.09634568194799024, -0.25250879032466206, -0.042501319338216104, 0.010584490076325557, -0.01950644221755729, -0.29813803179266735, -0.2192107756111721, -0.09968079397213621, -0.2784466252848698, -0.13908836957181694, -0.3515167314317302, -0.2359777693748503, -0.3228856591734861, -0.10697083077968633, -0.7799472750503812, -0.44958328567183364, -0.43245938789187366, -0.3875771277119253, -0.5839933019959348, -0.2654494251065606, -0.6317724168498454, -2.047445250150002, -1.777602389731101, -2.2540529227103177, -2.5553260509850886, -1.9768197281106101, -1.8624154666618093, -1.9827869615213838],
[0.1645568275358293, 0.21164871625409357, -0.2485417799307295, 0.5550462791083858, -0.38519890358785513, -0.09242633760684198, -0.11895486538937462, -0.0028919977430726263, -0.15726664635724524, -0.7440680737841431, -0.07160860588838676, -0.4607074638283515, -0.05266382337451863, -0.1344906157651206, -0.19996506070947823, -0.36957026600248816, -0.8166048430232253, 0.4811463279243865, -0.4752239497537076, -0.37070818356621993, -0.06820493739692944, -0.40783273384277174, -0.5841749665979173, -0.43356322900342753, 0.17335918830283037, -0.49254508613623366, -0.3855954268903809, -0.40639044517552736, -0.7267588396627808, -0.7584320156320505, -0.5081039635343298, -0.35761553821423, -0.24065938660090583, -0.624216959390353, -0.86951250460512, -2.956553465725128, -2.6637176970663767, -3.0013626861849434, -2.611583389490704, -3.330005029629171, -2.934463264386667, -3.107219709675934],
[-0.3410576889193585, 0.18249356651625948, -0.14184510000588899, 0.1101890112663508, -0.15667984971497723, 0.01803201420315679, -0.3820548515380104, -0.1720691721557626, -0.6519821854063634, -0.37186841116723107, 0.6126241647418784, -0.25847372511355576, -0.0420390274131535, -0.13569540782949405, -0.1980889987422961, 0.6403132512510019, -0.6780630685912835, 0.5103571183483088, -0.5874281663966033, -0.17972484627395413, -0.1254860680742475, -0.731875465965504, -0.39797342731815816, -0.17836858967390537, -0.4847518164036026, -0.6418870344742962, -0.673270427369012, -0.5040594242200891, -1.1771441019632296, 0.05165145860742872, -0.8962420413332801, -0.817782488958278, -0.7438736636366852, -0.33618585136203255, -0.685032197229983, 0.06435677835726683, -0.9511170614134021, -2.4651145180615552, 0.21892212871401992, -2.600451015952507, -1.823608415161544, -2.007567805172617],
[-0.22666842491576183, -0.05951154886748382, 0.15212571259271182, -0.2524277190275078, -0.02903964017557712, -0.0168016608877118, -0.2484283723881361, -0.24503253357758137, -0.013261547608720873, 0.024864392413076173, -0.01474818585693613, -0.2093633040682579, 0.14659537127048458, -0.26083323032858274, -0.0008475487080286366, -0.0382113446328609, 0.03207489886746087, -0.07266403329958819, -0.012663291613003417, -0.0062684378316525515, -0.02484285937803884, -0.10102593374071969, -0.06451677934643565, -0.1881287746677397, -0.3440623272445906, -0.30456548467141825, -0.152106987197137, -0.21800212943792244, -0.5683168371166568, -0.18974137882353506, -0.4644094302425658, -0.13525265381792284, -0.14600263372455652, -0.41900860063111633, -0.08270240078787648, -0.5716961815993828, -0.3488609190455125, -0.40208144625109044, -2.073983319331384, -0.6867819053768472, -0.5367843102119565, 0.025978186188341103],
[0.2642615734858167, 0.3721788294454494, -3.233072510932101, -0.7377881939294045, 2.5454086957083115, 4.777697246043359, -2.3890628405317, 0.08139843059408264, 0.6191331255160792, -0.8642046833979312, 1.3251803186405462, 1.4546416387768104, 0.2571523583319422, -0.557797199908841, -0.5321620653030473, -1.2099071870050753, 0.13935608053652046, 0.030484447790189682, 0.9830188477372724, 0.14270951409113436, -1.4586339445181902, -1.4002772602300524, 0.7604465816241546, 0.13006998802764919, -0.4124858138209726, -0.00171801307699663, -1.359459682806101, 1.7147917890564581, -0.9319291389754616, -0.8359982005296955, 0.7076921767946549, -1.2649269382960653, -0.9304751741489223, -4.9583766055288105, 1.3174006292468003, -2.1298476987668233, -1.8713721239606604, -2.034504176292881, -1.6677371648537922, -3.02670588234408, -3.8800632751399102, -2.3943680382028014],
[0.06248415947100943, -0.06452279513924444, 0.052973278636762024, 0.015771945851051516, 0.094178363087352, -0.22640116168873878, 0.10722021463845434, -0.1915369975293625, -0.05509379134455637, 0.14971622243587576, -0.26337011819287093, 0.14650676627525702, -0.17507917761445693, -0.029950365694570085, -0.05760486334918765, -0.3157041275078146, 0.19835835183096978, 0.04921681980705804, -0.06644113067632505, -0.345037409150769, -0.1312028507721881, -0.17005556052898693, -0.15633070909633748, -0.19305205061234143, -0.3504709261303969, -0.15755011420602358, -0.22452181961835802, -0.3424421436109577, -0.772572623616224, -0.4119261772212109, -0.5454600151421242, -0.3747094242349235, -0.35722855129320263, -0.2536985323223988, -0.7202011815780995, -2.0674627215364363, -1.752041223158598, -2.1355388169955716, -2.5953326670818617, -1.9128303757588698, -1.918260575659651, -2.1581883133844486],
[-0.011204776407512454, -0.09563042110101319, -0.01592203320839956, 0.24053521301622696, 0.2945227505850689, 0.1539453583648369, -0.04148068516409427, 0.08254679487607432, -0.2485701792876226, -0.13546392581136535, 0.07092779194857844, 0.009649003426607414, -0.08634846828184803, -0.15417495424117442, 0.04212426841754224, 0.0602973578873427, -0.1556860636115706, 0.008519595201791809, -0.06409233131202759, 0.04017716354164767, -0.12470367812665215, -0.2629762834311112, -0.20768871124698235, -0.2113321802075258, -0.2039521117947862, -0.3811303291823602, -0.3315812049163087, -0.15783436982287802, -0.6754288120300257, -0.42867585788200896, -0.5246133507496175, -0.403208451714398, -0.6828882204311237, -0.39936419905963666, -0.7205601506227414, -1.587459381163515, -1.5647644182195029, -1.9331419454306098, -1.726103932165129, -2.037579771339593, -1.4549310981061063, -2.0747911101574834],
[0.4793863931644333, -0.22803674005484953, -0.598915919652991, 0.14310572247821343, 0.41239591598695646, 0.45566308491853835, 0.03972716218909889, 0.33120778939061163, -0.2748067363213692, -0.60440080775689, -0.13050349672386055, 0.43506725037628285, -0.04673866558389061, -0.3295485547779411, -0.10142041352678177, -0.12589488835260593, -0.4381863557188393, -0.1985661704679863, 0.1797800143294612, -0.042831957408137764, -0.2981884096604067, -0.20760779853527694, -0.42696574503316687, -0.7036894105240101, -0.06295040336070923, -0.16766986057430153, -0.40140374498830644, -0.3358843689241059, -0.36174322808108106, -0.5878941050343506, -0.3197029810307757, -0.8310637239106461, -0.4439616632825529, -0.5022559403210998, -0.7890101181465364, -2.554212217695801, -2.1112526529212894, -2.3899262264661196, -2.3558139777069327, -2.645474982016129, -2.07743204609114, -2.583810429122895],
[-0.20231607020602788, 0.18320048822731386, -0.27424044292259875, -0.07673597157888531, -0.11341372965873356, 0.09390952046071388, -0.14078729866525488, -0.04045098421528417, -0.07644113236584077, -0.10969348357672642, -0.13271048609583347, -0.08481223482260274, -0.16712544694012743, -0.10272753443694312, 0.01144257561323996, -0.21515440296544214, -0.3588499881509247, -0.06706562426109716, -0.07388119673336792, -0.14634080702027877, -0.16485178193835412, 0.13830318442193917, -0.24138711436599877, -0.532854013950355, -0.42555732606036717, -0.4833515119273385, -0.2305450356965183, -0.34213895917510934, -0.31630067699944364, -0.5110131502140023, -0.10077069708682647, -0.3859556911286196, -0.4275246924392143, -0.5326896096746525, -0.25513216182004933, -0.5377294678039092, -1.213382362742448, -0.8710631997860706, -0.9016908422024993, -1.0284093846262947, -1.091541795713371, -0.856484744378389],
[-0.05486659604349217, -0.1000200119883735, -0.1275513741676643, -0.23409017998856224, -0.27307627075986324, 0.15440031084089062, 0.09482905205904714, -0.1213727542316487, -0.020971038239315672, -0.4197140591312412, -0.2665865426055771, -0.1178254463545992, 0.05102190360248902, -0.09192465292256533, -0.17999203371852943, -0.10543962431818145, -0.08663991497331817, -0.41374421988037585, -0.31016334433986137, -0.0972797483181999, -0.3647929425816127, -0.14230967369117384, -0.6124747056222846, -0.012401384085080614, -0.3784618154501793, -0.5317141402597648, -0.3459028452607705, -0.23181276606820345, -0.38849677815160605, -0.6215888874292675, -0.257822202232753, -0.06321980715486022, -0.24339917801463634, -0.7760878475679032, -0.6816673885345182, -1.6038132585276705, -1.1436062914475538, -0.9679672458531863, -1.6832118974649488, -1.3946444339063575, -0.815372515742982, -1.3332032521742012],
[-0.2914252338565335, -0.004303570625104549, -0.250057933901103, -0.33833198213071575, 0.03722437511365963, 0.2078046632546172, 0.019588306725815026, -0.09921751197490657, 0.024179455652709538, -0.14816999083560722, -0.13739704607379039, 0.10113196336209951, 0.07835333361805775, -0.034820050162008134, -0.23064804691164725, -0.05454379882432892, -0.04747068972272741, -0.30252345037811945, 0.15096226376718083, -0.023880856682933656, -0.07089648404415863, -0.024319072446379746, -0.12448127464647847, -0.10209740037886202, -0.32584922560477775, 0.026777723955914424, -0.25663067786381144, -0.2804494211015646, -0.7170114927900871, -0.3269965200985967, -0.4839117643344161, -0.14732986367823375, -0.5849672284195875, -0.7249125038930609, -0.7159984032286122, -1.534153582988061, -1.5530684726873736, -1.792876374968993, -2.478633781339916, -1.9796361225816794, -1.7454848757163273, -2.0817088331117266],
[0.16897091671734035, -0.23042741484960264, -0.0694323888150255, -0.21422608152981848, -0.21700659500095365, -0.11359313256003693, 0.13215904074692603, -0.20530880880524988, -0.1414335097474967, -0.22759718934089931, -0.45407547665174613, -0.2674301923666692, -0.12130000710482282, -0.13873434730518794, 0.06325570283057204, -0.03322863520065524, -0.3168666346436702, -0.5245465107226098, -0.17179303457567974, -0.1938051298377351, -0.1613446667191555, -0.07657258703619167, -0.18895889901715268, -0.033955737067481055, -0.4788103395669397, -0.24415044271047967, -0.3154752703591714, -0.214116746701569, -0.6043263245810125, -0.620543956680521, -0.42747090155048806, -0.3647006637939856, -0.38650528058693834, -0.6897003461524596, -0.5563183610561314, -2.4112533611899325, -2.03339531335632, -2.295648296186395, -2.4241375640272116, -2.2835192851767676, -2.177178418349539, -2.420945197794368],
[-0.31454606363084564, -0.3657816633002574, -0.498702460869793, -0.5879760436842587, -0.29478907528913284, 0.12611891971871914, -0.07027055072181002, -1.1497337602140068, -0.8518828093631903, -0.8604701346935093, -0.7033016469837509, -0.19950438768545112, 0.018690832749532028, -0.07034037483341786, -0.7302625464397191, 1.3710690275754014, -0.5515774391376796, 0.8956322935165186, -0.2558170055551289, -0.37077673755327534, -0.35958624146867557, 0.039391571754255617, 0.6568975379348817, -0.1893710923163832, -0.8838294194672743, -0.8151289859751115, -0.3610508375804058, -0.34268866526772473, -2.510286930114454, 1.5519953213285735, -1.2354367758012987, -0.12488183452430066, 0.25438797632583393, 0.24006938729354438, -0.6588619097996528, 1.4317323259168582, 0.5822176376260865, -1.2498406230463979, -2.200433445304635, -2.188555633997189, -1.0622854995891124, -0.8093337978788336],
[-0.06238734383727175, -0.03138946867703862, 0.03342118432065514, -0.046104875634280326, -0.16321310059646088, -0.019377580999718518, 0.19174625521792327, -0.01981890099339983, -0.12518428904962303, -0.3748278398276819, -0.07616782996170586, -0.2183550289742564, -0.05513717524929036, 0.13620300028503057, -0.4714181877950919, -0.19574664065210245, -0.5553233235642713, -0.16383654852075583, -0.4039431038959571, -0.1581034104606447, -0.19644345102765465, -0.05135665460819961, -0.36627243987391256, -0.342317781200594, -0.2949438806610403, -0.2380652198009894, -0.3448000891256153, -0.171273281416782, -0.12771332268586222, -0.6194920024000534, -0.026595474136098186, -0.4507164423207672, 0.007403025207519751, -0.6181858994769157, -0.5538835810392586, -1.1073742063342193, -0.816263829333722, -0.8231018109547167, -0.9411129346238871, -0.30524672673332937, -0.9026111225079806, -1.1578657306814595],
[-0.269450428131127, -0.044666989917402186, -0.34037371055859833, -0.1882131684102853, -0.09239139308168443, 0.180240369268792, 0.4482095704379949, -0.04479705209158868, -0.03389684533627775, -0.39364630445612764, -0.2110395599075825, -0.14326818967198962, 0.2541982630811065, 0.23483474175762042, -0.6328064910647524, -0.4008447352323912, -0.2991158061479284, 0.06172839221572085, -0.2811550087192292, -0.045818777330894844, -0.10656348097262672, 0.1569437080024554, -0.14142357367630712, -0.0056969090808194536, -0.5319730944464145, 0.057944353407351996, -0.03859524916691405, -0.3162376617782031, 0.010404403101319964, -0.7398037601046994, 0.12943555080459343, -0.33660970918180627, 0.11837899053484309, -0.5134470171632132, -0.291512093262646, -0.3613254189835862, -0.5439130136213903, 0.08873888812305164, -0.7615463139306008, 0.11487555690568203, -0.22241840934609747, -0.6842609452200575],
[0.1551177628478855, -0.16458431984425867, -0.0795198377703692, 0.002684565802673643, -0.018450982178780634, -0.02866331246398549, -0.15983080097322214, -0.18661689087232503, -0.06658336659864827, -0.11640893947479762, -0.15102252130425867, -0.22229925278396453, -0.1721953784074239, -0.18555349128088444, -0.193826881001375, -0.2410432721110694, -0.1128351385510327, -0.14323469404559844, -0.3103913679680124, -0.3766148913833733, -0.2960000317956998, -0.2582817123703716, -0.5793992364462756, -0.4430532546008614, -0.34569633352696766, -0.5421470646200935, -0.20256418641322907, -0.38117453395838363, -0.7165079878083277, -0.627791503707851, -0.6612961288085544, -0.35323487151959215, -0.5863491982133776, -0.5439760122602451, -0.6682955314230503, -2.132488640837545, -1.7032900992088988, -2.0145202664038315, -2.1119079355963177, -2.2475135882411816, -1.793471558734712, -2.055189007138003],
[0.022747515444180508, -1.14396518864877, -1.3428375509360448, 0.11727189464664033, 1.0795571631397223, -0.46586285235576685, 0.2775418279707735, 0.16474949815369785, -0.09202690000453466, -1.2001968228925548, -0.9411781667531401, 1.8110649092328182, -0.692958277807473, -0.16804636907590398, -0.2436070159250226, -1.4661167117105802, 0.22041159020260578, 0.6856338360193657, 2.892517733674116, -1.1029123467323543, -0.2947506803293447, -2.210142491971868, 0.40925481582825735, 1.1603318390871271, 0.11700361262841336, 1.134040696301647, -1.295658947761718, -0.6993059188195965, -2.0075308200148037, -0.5618440501838128, 1.3177821645937287, 0.07682739150826083, 0.7281264721617864, -0.9074667435896471, 0.027690026053012257, -2.3347877006456024, -2.0565465070309985, -0.22023356816821266, 0.3321113563622767, -1.1818178763723408, -3.0096678301587176, -2.2133467913177594],
[-0.9758980235977289, -3.0354466361279946, -1.6633571254088368, 2.200271207284549, 9.370523644265175, -5.138226162690197, -0.6345604269780811, 0.3022972595637566, -2.6717451387948064, -0.2311173872078633, 4.087099240400328, 4.439311177629539, -2.429351522227118, 0.9834880105130197, -0.06452676153350678, -1.3512989260664978, 1.8784596692852145, 0.3988495517733747, 1.8594892780069612, 0.8708571000093339, -2.05829879278165, 1.1987335597003907, 0.15978381116288734, -1.1576469011298853, -2.4307850683398082, -2.7814800889918128, 0.9874599795453329, -1.2069925284249157, 0.381837644892135, 0.2607959883299215, -0.731464678462036, -0.680059899356084, -5.090085431986473, 2.797886619566069, 0.5875365332438812, -1.7974397443812637, -1.3464778927850944, -1.9207356674029075, -2.4991697289623502, -4.884226023320265, -2.4337025564721166, -2.7803875717876774],
[-0.13201093373417294, 0.08990488850601108, 0.0723952277572632, -0.34423435165431054, -0.16117799488779025, -0.008686830960131165, 0.14835981388874506, 0.038616092141502426, 0.10840741557442234, -0.07070721103254728, -0.15920774781885652, -0.225793710571991, -0.1319079670327562, 0.05041265907183092, -0.1799470039347729, -0.08682624355135492, -0.05346146045607926, -0.37579850358460604, 0.011893955914155098, -0.07586213031162897, -0.0484087466975916, -0.09442740386198648, -0.06066623981374785, -0.35366162691140385, -0.5249183160084664, -0.17349224887357323, -0.2697797541201022, -0.32443373681584325, -0.6901644044092455, -0.5404136412597316, -0.40501141022506354, -0.1807312978180514, -0.3940049011024536, -0.5420447170502899, -0.6339385192043374, -1.7621437169283642, -1.4950060343482325, -1.7399655664099318, -2.328609266337959, -1.743655180127368, -1.5921643813236204, -1.934629182587305],
[0.3678433310131774, -0.45382319481144145, -0.1505902437224751, -0.6927380795682277, 0.10819271373498662, 0.32384839800155246, -0.03868847694654002, 0.07468541624419632, -0.3148828900000168, -0.21804099179632536, -0.5432634984585043, 0.10175217229308305, 0.38939698329954137, -0.17999399143056014, 0.18060778627200866, -0.6457465098715245, -0.07963603262886922, -0.7923071146718059, -0.563640224791791, 0.030739561857296333, -0.09240253118828919, -0.2691633553576383, -0.5087361972835842, -0.10683479303223642, -0.23660052154632003, -0.5951344061497047, -0.061298172041561926, -0.40192966256610463, -0.24760743179260172, -0.8495610446457768, -0.3080270773427703, -0.34892057421246514, -0.9505055555177809, -0.5246961592597528, -0.5471882844554897, -1.0580694640603252, -1.125204954704228, -1.3088093861667365, -1.7395140781167233, -1.4716429890552312, -0.29359881613340316, -1.486716553640722],
[0.09225602143983072, -0.15674682086664957, -0.17585293644021985, -0.30985502748943133, -0.11608922998705577, -0.05869000326678433, -0.0589731442144227, -0.21109540024046433, -0.0700175536670454, -0.1518871518598696, -0.12976305920339984, -0.3316812859343427, 0.049342737085076104, 0.016077289761428953, -0.08969988622795526, -0.21051491591185265, -0.21016919253802832, -0.35767862012842516, -0.20484776805779897, -0.17083998132062012, -0.32533019044053135, -0.1308399868771792, -0.28606773391055346, -0.025213254886406463, 0.05223961777619295, -0.3921584702382121, -0.03260846388784779, 0.21779511100630178, -0.4935046917788943, -0.6906056094106049, -0.37700792190630106, 0.3880365131601743, 0.2649528319912077, -0.6150453767590341, 0.0025465834278497923, -1.2347840947434083, -0.4328638386476419, 0.1543510625867289, -0.49462921561159184, 0.4595786111778214, -0.36604253296720063, 0.20315117447334777],
[-0.07995234508997401, -0.04364701557282157, -0.15854072763705412, -0.22735772595374368, -0.06491152027875667, -0.048636973016035034, -0.26014302895086283, -0.11152132593450094, -0.15320836090299736, -0.08712828667667241, -0.015837594232578397, 0.014524651986503094, -0.022231959983226975, -0.3721676927824507, 0.019083032993058018, -0.18533110968400876, -0.20119310815191316, -0.40366277950312995, -0.24364466571825172, -0.6081833150701165, -0.11890329142817199, -0.23854149539513678, -0.4337444760239626, -0.5037455252607933, -0.6277258320666946, -0.1272874081933842, -0.6653047231443207, -0.42323486620936607, -0.5841496557864587, -0.31485275414704395, -0.5907992246395056, -0.2845376857302286, -0.7036576331867154, -0.43803740852874706, -0.6524399318227251, -1.4035841306260737, -1.232662034751673, -1.1791676997019847, -1.7894090208273656, -1.1584124771520694, -1.2317383971066682, -1.4402141538080266],
[-0.0361861463940321, 0.16524513273315597, 0.044764370762532875, -0.18541657574794249, -0.027123394653919435, -0.041734609896747106, -0.10538229222569284, -0.16782878573836738, -0.017243604483846226, 0.039694548006021, -0.14164251170247144, 0.0927796580529788, -0.18799228864035128, -0.15932393842554274, 0.06731504910362929, -0.2616160518221281, -0.05189327448699458, -0.22878394249174117, -0.056844044692947245, -0.17362753147339646, -0.21526330604503893, -0.23908436573906602, -0.40976095683234176, -0.10240497510517167, -0.3305892005178168, -0.004304826863335106, -0.3793359717807508, -0.11217726361598684, -0.710041353559223, -0.41099589438498363, -0.6127018287545878, -0.11462585059176614, -0.49201320182654007, -0.3410791104253733, -0.7912465683515628, -2.3854756509116433, -1.9908791624382853, -2.2219319875040022, -2.743929944546006, -2.077567333723292, -2.0980724530966666, -2.316953017358143],
[0.06689512421302582, -0.32493469677930764, -0.04525882716717499, -0.19460721608460083, -0.1766851975144251, 0.10015905204331091, 0.1446372176463529, -0.08324871959752324, -0.4314441840105577, -0.07961764210422421, -0.5819008347970643, -0.14657622626574043, -0.031242530993929433, 0.0484880270823798, -0.1283303715625658, -0.07659568391955424, -0.048474738737040834, -0.14011191043554233, -0.4695017406676022, 0.0005912905907942654, 0.07716566850966548, -0.4521347939647734, -0.4018668819099469, -0.13435277988274888, -0.31732703147571045, -0.648389551927221, -0.37715192225835764, -0.2432600427503551, -0.6871069763179151, -0.3152636575022756, -0.10954121037118944, -0.2776142684204385, -0.5336308064135111, -0.5687681709170711, -0.6381595423183374, -1.5852606016449413, -0.9978543206919992, -1.4494698683116234, -1.7482310723355552, -1.3499721632822699, -1.6045934744244157, -1.8215376634135252],
[-0.051441710966833254, 0.260937712127785, -0.36009125502657097, -0.14325440740939555, -0.23410735095454785, -0.1423298102221778, -0.36532322435161985, -0.24262595378273658, 0.07770560534652485, -0.4383102191119863, -0.15162311620510985, -0.2247478767958425, -0.025082283436847125, -0.2510690511767354, 0.015351466869008484, -0.4267904922788371, -0.29084810666017913, -0.6537600661502605, 0.0788087979342347, -0.4191318017938623, -0.4134014312335427, -0.3487401386273093, 0.05796847165544999, -0.38193121092233745, -0.5466893902322022, -0.07945885435424627, -0.2592103452393773, -0.3742289759830806, -0.6366159243122379, -0.605454769766981, -0.85357062635491, -0.25969930088972865, -0.47167369688576477, -0.6773830824773942, -0.6889417948763068, -2.0238208509693383, -1.6683223992997365, -1.9308545516316633, -2.258209480716249, -1.8837447760608352, -1.8631277474419297, -2.029043253045087],
[-0.08270607760148319, -0.6584546349783464, 0.2992566192095406, -0.3797658407452005, 0.09890310684684815, 0.29470469552382167, -0.3502232330552351, -0.6095452699843423, -0.0318455826303162, 0.37732030874548056, -0.45474693276175127, 0.33871522086049094, 0.13358347059770748, -0.25754142542089864, -0.5200687298554312, 0.32396212939253166, 0.21433815018347874, -0.6632563294713657, 0.37733661513002664, -0.2764006588471298, 0.050312000970047954, -0.3033893865360314, -0.5263602721942693, 0.011640755092635135, -0.4243813042380017, -0.28160844133037777, -0.4034495421414799, -0.37212353387771224, -0.2886652124615866, -0.12096083556841258, -0.32902318188767876, -0.04067230302848225, -0.5164454254353389, -0.9980411800138628, -0.8146582852756048, -0.9249737476248743, -0.8824641351762613, -0.4244571910136298, -2.0212167911973076, -1.5323615828630586, -0.8217167116780602, -1.2864795211715512],
[0.2481197139461848, -0.21417007124666831, -0.0004024954815845505, 0.5295181166587799, 0.12644786298155186, 0.3200591979873386, 0.028991186645814793, 0.2629912264390641, -0.2278645606950375, -0.02223131627465093, 0.07459109094897, -0.13812000456576737, 0.18371684314469647, -0.07721693778587066, -0.12037198439625947, -0.3009574680298431, -0.15733040456383274, 0.13328349483128443, -0.21191364500404544, -0.07111112675438669, -0.14732634242584058, -0.4591047270296095, -0.6442297727020233, 0.05166568418687569, -0.17504932432666115, -0.13870736587215834, -0.5078272292781801, -0.11264888459657278, -0.17930016577757021, -0.19614544955215002, 0.1268595448163143, -0.17202168413295915, -0.0790245427958553, -0.4079507058186403, -0.08521659877260279, -0.898433584939286, -0.5690802304133241, 0.29738094654431335, -0.43334641007744223, -0.7013352893932469, -0.5350873628519607, -0.56389229669869],
[0.5184281686402692, -0.5632268570527033, 0.1863943156237835, 0.47163780095728286, -0.3438793148800591, 0.19051330217912704, -0.17162061714235163, 0.47040575670458384, -0.6055937063685679, 0.1527477425253678, 0.06524966617163876, -0.26129171032526455, -0.12587126901159834, -0.15319719479506116, -0.10245313640307659, -0.4431930345354048, 0.06162505966548307, -0.04636630210202972, -0.45545129560327635, -0.3621810771478761, 0.21953918808814218, -0.5298935178479067, -0.6795070212550584, 0.4147814740459514, -0.14167568065227779, -0.6248944496038225, -0.6211840547575201, -0.2330515655266016, -0.7513053798300403, -0.25859714179519744, -0.3375442500685458, -0.33641635807995535, -0.5247543925635417, -0.34989419414781153, -0.4501107172825401, -2.7331400682914486, -1.892281156879237, -0.5840083841398384, -1.8809294481145973, -1.129340451901545, -0.946937887408369, -0.9716140263884412],
[-0.10427276434371335, 0.1334905409073572, 0.715613036875259, 0.5477537895587811, 0.5026292275665842, 0.49182577626157864, -0.2985068061793842, -0.22517412351019325, -0.08327592174885907, 0.30898010127338055, 0.3379206316631177, -0.2186237275344243, 0.16828269660477566, -0.2862327823844508, -0.22696505778641402, 0.026372815301239957, -0.33938493716648366, -0.04990135003506895, 0.16584229636101427, -0.038962807856329836, -0.10831319301085082, -0.11229121882602361, 0.5868984972779847, -0.2347978079328366, -0.07743028291553758, -0.4611086723060079, -0.25790631614522513, -0.06969522705752482, -0.8187688435011038, 0.5482495302368913, -0.6427365029385498, -0.6466933770912747, -0.39065890102128226, -0.5151760968004918, -0.1304740332089904, -0.6176496869368161, -0.40882274773891425, -1.2086841471214336, -1.7128441591809929, -1.8086302234195923, -1.1845152996039532, -1.013036955508335],
[-0.012419996982885543, -0.14287920037151014, -0.0716664229161558, 0.2611421423153867, -0.11514320696361502, 0.1152900491149719, -0.034438658895869646, 0.11422878513856838, 0.000482888730747127, -0.10795467905956299, -0.09650526154559737, -0.2167711345937627, 0.06192351908885015, -0.2793680126037893, -0.18019504145180085, -0.1995752116612158, -0.08966406572645216, -0.0276363388115872, -0.17978815883913107, -0.17838445774497746, -0.2525451802224829, -0.024354171195172383, -0.3125868110298884, -0.1153685332020925, -0.35632566238428287, -0.35742871755131433, -0.041602644504486426, -0.05450232069346885, -0.4258796106121973, -0.5402474380402692, -0.354319822441031, -0.5127369480280064, -0.5354664564792334, -0.09095697491030118, -0.4771709324374564, -0.8682120026180663, -1.0615613380955202, -1.0159137410809433, -1.2549380193882897, -1.1244758052333914, -0.5279756593136695, -0.8962156287784272],
[-0.0723848847522808, 0.06641233911438756, 0.04930434625174776, -0.21650772706234359, -0.1703676424979954, -0.10503787070372603, 0.16021965567824448, -0.23016386693268992, -0.44048831394438426, -0.2103661869176538, -0.08750233626044646, 0.13508662205353678, -0.10455468478021988, 0.05288531424865169, -0.3433186480201114, -0.4728000508364285, -0.14724200163491624, -0.5078644108141905, -0.21583333850394024, -0.054690432816569504, -0.0642813861436432, -0.39932065652412074, -0.42728189150655566, 0.0645394268274885, -0.4637889785068553, -0.21905242409548473, -0.06469015669033217, -0.36016911602921703, -0.5923194039681391, -0.3107180698903396, -0.08980673057552123, -0.5789280531921424, -0.727616984815044, -0.4864312882986397, -0.6633417804116062, -1.0627130485647647, -0.970262106452442, -1.0276680178709539, -1.6370863275702434, -1.147864836644132, -0.6905693307757997, -1.5921341011600176],
[-0.12719821117594546, 0.11490008830742916, 0.9577393730107722, 0.5810296080893459, -0.08366865511625066, 0.9612234218677076, -0.6632490636367397, -0.06396009316193152, -0.36244118073283876, 0.04747594593338559, 0.5271096377305853, -0.1614410454773964, 0.47380688111321406, -0.19764774134838312, 0.07208845084449124, -0.19897207964508157, -0.10240635599460732, -0.1733828258871928, -0.056900327045063606, 0.15456879787469247, -0.26791858373343563, 0.24118002871418362, -0.09305046395632914, -0.5232194707718058, -0.229773084181286, -0.12554503874881834, -0.36862062998043116, 0.37840911409928735, -0.8534244978739701, -0.3119269669868391, -1.1199262146823368, -0.35274144202768665, -0.43958977240955954, -0.8434379372398149, -0.44958644614229604, -2.561827157747071, -2.121131802143317, -2.892151096525538, -3.0964429405722083, -2.5978118112034534, -2.780140805427117, -2.2501038327844984],
[-2.624532537340166, 5.591682049479869, 4.893506410561171, 1.0081321229916547, -1.461497408913772, 3.9896685513206798, -1.8858655137515694, -1.2586002135861096, 0.8383438048905758, 1.117023822493317, 1.3714991962772738, -1.4754757429735101, 1.6157851968559205, -0.8421758749219206, -1.0402045018726893, 0.7318730273934423, -1.03312451061301, -0.6022394518154098, 0.338032966456222, 0.51363532493175, -0.9166927251180678, 0.19633864799313552, -0.8329833745331404, -0.6192016957096722, 0.11616723884390583, -0.009008502412150754, 0.6045505373906775, 0.3725016665308917, -0.7440531180008414, -1.267154990705343, -2.1598305663147443, -1.035097603473432, -1.1700008432920401, -1.4909441905722203, -0.6711597391276971, -2.8645891604042353, -2.6702795870895257, -4.219015702971476, -4.487325729060702, -2.7022144432265724, -2.694346018507385, -2.406108974029815],
[0.2571391015642269, 0.07803684692750175, 0.12000312294428339, -0.1640763231155354, 0.10976845714684677, 0.014166904049968282, -0.11131891761362693, -0.060744695512813626, -0.3222611807006494, -0.050238415353094326, -0.2702870974861832, 0.031838926741609645, -0.41453820102918015, 0.17610881925165275, -0.06612394680230703, -0.10991309386499037, -0.32167118175266557, -0.382325441749077, -0.07970119703313812, -0.07480542023871278, -0.2658937795410817, -0.2782137152899054, -0.40586788907716237, -0.37696178098202504, -0.6763090710765686, -0.15762898729450314, -0.7076510610166064, -0.4284024872016087, -0.7030906110800341, -0.6135127930963002, -0.5318457946816486, -0.2487996186397036, -0.32436387401733596, -0.6942714538703331, -0.5904002114467096, -2.379079231257568, -1.5638315992054137, -2.0279335193614525, -2.428499134152136, -1.941482725881673, -1.8574377205980888, -2.413501377264764],
[0.05866479577399469, 0.08597549087217433, -0.09453566651379224, -0.3524529217916734, 0.0726598754969362, 0.0180351233677461, -0.17887153423727176, 0.02744614369205146, 0.028587239885925472, -0.08828743940321179, -0.19388909979276842, -0.18670740730129193, 0.08073006390528963, -0.01625453496782955, -0.026633520277458324, -0.207593782512937, -0.1789701722128715, -0.17436292090948963, -0.2153237364677581, -0.2876138797015895, -0.11725087218333109, -0.18035800877907107, -0.32356535846951506, -0.16378506580963909, -0.3891268962557885, -0.19796006194989665, -0.0996955694646626, -0.24029888706398506, -0.8531744842221285, -0.5244117647274985, -0.5461021982297319, -0.15857004354765894, -0.5049625973742249, -0.5280047903284101, -0.8444108609121938, -2.7095087873153454, -2.375403548132812, -2.619378847517232, -3.0543060596680878, -2.8485721197594374, -2.5063026569750515, -2.6935612618126354],
[0.6550284659910403, 0.24425761621183129, -1.3502725468274164, -0.016657159962363773, 0.8675921075629418, 0.8391049549733359, 0.39711595771681324, 0.23287169777134206, -0.2184153123866928, -1.2003587069983361, 0.14616878193794253, 0.19827686806156683, 0.14181388702882067, 0.30076736155115036, -0.05766580222433517, 0.00549975132937129, -0.5508243902280182, 0.04125405397388758, 0.08049409325391835, 0.01699609038898754, -0.08489099658992606, -0.6173346797712347, -0.6789238844832034, -0.5744685492024436, -0.42151608645829863, -0.24424798151111243, -0.510922068473707, -0.3622094501592785, -0.19648273546540904, -0.7027550546046452, 1.324412873954664, -0.4947607249701461, -0.6089448896376831, -0.7341139180320394, -0.7223231056151848, -0.94814028810437, -1.0880188110251976, -1.6183352313317458, 0.9594486786337711, -2.6666053542925394, -2.132597792343321, -2.628610648719728],
[-0.1749344536018687, 0.21877719306771792, -0.31986000079230664, -0.1587448229762807, -0.1629572788922897, -0.05789586740451724, 0.19581580604817597, -0.05881068714743901, 0.07390274292733028, -0.14640737663089734, -0.11723156580529417, -0.1782589568278412, 0.02413631015020087, 0.13385595547854254, 0.0028028887907674827, -0.2087466963680038, -0.20637640676289903, -0.08717437589762635, -0.11171234077132869, -0.15409482470913605, -0.07944580091199022, -0.21263304272049968, -0.3317959771045521, -0.1904606766463768, -0.4012372992420545, -0.33699442075563196, -0.2508157035815891, -0.11531697574956953, -0.7380764959742034, -0.5712030679599003, -0.5462973849812205, -0.28891388487516384, -0.34397413820041356, -0.5596765017044768, -0.7705520721519629, -2.2309714163338072, -1.9838313034034647, -2.4345189080011904, -2.631896898802657, -2.3979989860623276, -2.2493118857699534, -2.233675565122042],
[-0.21701442585662759, 0.11602705671744407, -0.04569066603216372, 0.007331470336114442, 0.19453016238900128, 0.13946215290841213, 0.23933456620457746, -0.13090121251117912, 0.07185399656554985, -0.12564293174266522, -0.0936206255822648, 0.0585387240890462, -0.08317874349450001, 0.09289127667701755, 0.026827006100356158, -0.22785469802453598, -0.0030227772195409747, -0.22265291320591557, -0.21452061234087758, -0.19357565229640117, -0.0633114948477068, -0.18197009668198957, 0.0009287841835130784, -0.05990479195622188, -0.32214117930882796, -0.0895175284314261, -0.3075536592492144, -0.30691587173106283, -0.7270509130304984, 0.3561226520856352, -0.34814505775214283, -0.39021948954339025, -0.2827590247868784, -0.4927912465289446, -0.3597792925959002, -0.2701276436085974, -0.15995656938750974, -0.8375928510576232, -1.3244460910171365, -0.6582445728857812, -0.9882927530810866, -1.2990897807942157],
[-0.15957427152604003, 0.2870872774619076, 0.2601020847814625, -0.041899018193596246, -0.2606189385096772, 0.0786008253721219, -0.09520117171273171, -0.14139590487639472, 0.044064012108593495, 0.16493977214470232, -0.07801982210349094, -0.29736985512606107, 0.01209570753607114, 0.07819595922965727, -0.029642521007805386, 0.020303558724322133, 0.08348736820027969, 0.013417759444703404, -0.06113359844977061, -0.03561348533975233, -0.2011582319913978, -0.13587205634733382, -0.2528834278971684, -0.1396600308582738, -0.22757536819985164, -0.22588666504172825, -0.42948414379224575, -0.14926933441482343, -0.7355983282730455, -0.4050053403521183, -0.6889370121361019, -0.3035211162112888, -0.260290538576497, -0.5635668628509961, -0.6358074374066652, -1.9384664620045, -1.7672860697276058, -2.006355153042759, -2.5502608619618052, -2.012337294552732, -1.8904517260823344, -1.859742724088788],
[-0.15674920239031856, -0.07870020993284335, -0.11691817160513482, -0.13798952478814588, 0.07417255894849337, 0.17057415437464143, 0.01214703949917992, -0.20399693469662086, -0.07381687848926485, 0.24738303116057275, -0.07631935280198807, 0.08457492524579234, -0.10088463175493237, -0.1886590428438134, -0.2976997797699585, -0.5292178312791, 0.4523925237149891, -0.20552247901928586, -0.32055476884650685, -0.4254630377919037, -0.10784103145885704, -0.49395086986130055, 0.13676665620894313, 0.24122158226604917, -0.34589475901559535, -0.025721260872095694, -0.39082958630161435, -0.4806160429481098, -1.1165768102786362, -0.12590447596166954, -0.5009047184036637, -0.5922704553703282, -0.5090460717836841, 0.1400334768782401, -0.7419033754975451, -0.6137176388279816, -0.09354011151590838, -0.9255040078091556, -1.2967407307940049, -0.8170705256649421, -1.0519205515716177, -1.545456958146928],
[0.17838582019777358, 0.08182521819369142, -0.1081307711528438, -0.132627053144558, 0.1566549113706885, 0.14653500131016478, -0.07993737541313622, 0.01229535544759765, -0.14604316194693903, -0.172150364148421, -0.29537360471538027, -0.10398695002837269, -0.10094233663159052, -0.2379441174055839, -0.11279466217532645, -0.08849300003803708, -0.26810809382670564, -0.15403284854586596, -0.09721317042991925, -0.14957393529622323, -0.007697118251521223, -0.39354393698799306, -0.2735489838142422, -0.08557327476788937, -0.19365440001396636, -0.2477945890931161, -0.47028153556741475, -0.19634212753324856, -0.7092649244735827, -0.3954444714265006, -0.3920090073050758, -0.3950281812435657, -0.5350112659611191, -0.5135832197425471, -0.8771051529568962, -2.829201279018126, -2.5392989369797494, -2.8422819941762563, -2.798509763338332, -2.9746176911106414, -2.6661160335477594, -2.98768815508316],
[0.026221285573926063, -0.24305876389808287, -0.2079582912511661, -0.2523367700886755, 0.2052971229119644, -0.2728263571394544, -0.11515166790980348, -0.07552797078211104, -0.42807759674138746, -0.198836735629109, -0.27564120953865695, 0.24193241334740814, -0.26876674152840785, -0.2515831205606077, -0.1802279101038682, -0.5052487683184614, -0.3158456954794313, -0.617785389045874, 0.041683756132307855, -0.5792196053794936, -0.44399105627251384, -0.10288016092283808, -0.5373146448590297, -0.10591958721814762, -0.7379576580648971, 0.15687042247404026, -0.03680793012008037, -0.33212823324591967, -0.3914647804916011, -0.7028739352533763, -0.3443224945716896, -0.3254333408779291, -0.3079013606494512, -0.43340327130948497, -0.8163490112963563, -1.7160594208883992, -1.3022295646101918, -1.2361580590207375, -1.7402346465470013, -1.2996398245891885, -0.9945146329935316, -1.987592603658657],
[0.7899380598575773, -6.861882017389881, 5.067736725571482, 0.5639185607713812, -3.6243996562146634, 0.873533345931249, 4.263563886416881, 1.6067853665113812, -3.554016204770893, 1.058848026592259, 2.1402366183435495, -1.3574086693869754, 0.481096873036584, 3.129433924575932, 0.9027507902578358, -2.3328778189415336, -0.888493827445629, 0.26032505322882, -0.07982822008560238, 0.8831718150003877, 0.7247602565912138, 0.1500619614335988, 1.351622092953315, -1.033259873240991, 0.8973156449005896, -1.36124782605022, -0.379140052059224, -1.6622322531841975, -0.8703126689628533, 0.015561922820349627, -4.4877659634502844, -0.37069806113860304, 1.2638408671065204, 0.37495425601889054, -1.8007319679028735, -2.8422784963562098, 1.0491263922507061, -6.956052753414728, -0.30919204731825434, 0.7914523013917799, -0.5504437314714106, -4.1219148927771005],
[0.10988546590401499, -0.06895670259049362, 0.03300900081323142, -0.3878243239808736, -0.021783730181472095, 0.10125898956758374, -0.0064329453555004865, -0.018342271910229538, -0.16190583679637802, -0.09830524315206167, -0.37298172503863536, -0.005004085587264112, -0.13068126814300873, -0.1720424941786157, -0.01294398001732066, -0.2671601422276286, -0.14765714979265893, -0.0992210539568659, 0.09675273387584843, -0.0468914015729592, -0.17762357679878, -0.20595135574823326, -0.23745936014344846, -0.10570425566197383, -0.46789257777135784, -0.180676521308931, -0.15455636421207053, -0.3321529740840977, -0.556700543992766, -0.3922341185244757, -0.4649711964313139, -0.0252750410328201, -0.35161858767373605, -0.5564215612354059, -0.6431183353515637, -1.7349630572391868, -1.524576875911142, -1.065558758305474, -2.303049391965729, -1.5035827691729649, -1.2857667424978945, -1.5405644397886646],
[-1.6270638827188686, 3.714831321974419, 2.6685277448440248, 0.6458005497133327, 2.1865189623550285, -1.300022950270943, -1.6529376077815512, -0.09210273181803573, 2.4390894800463685, 0.2433221767338234, -0.31282165299622894, 0.530594941089332, -0.5887349888973286, -1.551775254319519, 0.47909570557242404, 0.684709928638949, 0.6478000904641424, -0.09647910949684424, 0.1508240501479412, -1.1177891016333692, -0.473337182833121, 0.35457783956454314, 0.15087659393107355, -0.5250629631873936, -0.181889655327788, 0.6385405923897167, -0.47745412211899807, -0.4064845209362915, -0.6917231725524585, -1.5126272677814228, -0.4318287694532303, -0.6882691780402641, -0.17234890086830193, 0.2956604620361619, -0.8476149485389242, -2.134029759464182, -2.986243746495932, -3.288706998359296, -3.4958820381583475, -2.4756105161266175, -2.3118218550551286, -2.084820099726138],
[0.08917969318247616, -0.26321426515553165, 0.4007786597675641, 0.11825079122611204, 0.1890146154247517, -0.07551720613413558, 0.27381316375323755, 0.18161937585156948, -0.41499414687874875, 0.13979896042646298, -0.5457651260543397, 0.009137111915759515, -0.09165115004530029, 0.3857874922485103, -0.09068361135914209, -0.17655046038813907, 0.05860304459590846, -0.1922181700015041, 0.0224468272139417, -0.24424576779978358, 0.08076195239077429, -0.30618696752581703, -0.791609024951537, 0.3277660264119771, -0.5253776614556599, 0.21232010626049982, -0.7048694253801118, -0.20499969013108318, -0.42249042009961424, 0.42458939586662525, 0.010320308161923464, -0.7923883090784559, 0.15728932809747756, -0.011504812204430323, -0.17287782035004048, -1.3536675208329139, -0.10782105529869651, -0.12873991253999187, -2.015304858504847, -0.23967434770102528, -0.6203048372417113, -1.4031709010047788],
[-0.004102951122672002, -0.08267141660700976, -0.06166177224788122, -0.15943074759621712, -0.03538035401920923, 0.01958209424010969, 0.027964701972532983, -0.2072737018486234, -0.12248784813494516, -0.12708884903932363, -0.3047142796292587, -0.11234477102984855, 0.11684338784428724, 0.04656307116692879, -0.0525754590793734, -0.4536908095745574, 0.046633013448697636, -0.6201625306561398, -0.28933531905404414, -0.5068371132739194, -0.24569169049739908, -0.06003311691671903, 0.11919635447357466, -0.49957715342005204, -0.6186162713723049, -0.3129148000455468, -0.031867728991237514, 0.06953578119574232, -0.4391395940801058, -0.6020178195263058, -0.36458374888902595, -0.06219087728555052, -0.6312957482611259, -0.7123230560863135, -0.6191192111120427, -1.367403815169, -0.7068793782971452, -1.389529193381341, -2.3580287822903365, -1.5292347107714395, -0.31052280400361004, -1.5803936581255247],
[1.2519240856996263, -1.929744979584927, -0.04360085905810981, -0.7723326353509774, 5.2285286036308305, -4.3963449858248085, 8.753503666715632, 0.8185080417558936, 0.11061341520026091, 0.39529794777524807, -0.6693456633839876, 2.6208292281404044, -0.392563170880618, 4.978532015421452, 1.1661155908620198, -0.034944137008312816, 0.9387931457147976, -1.6308588517253577, 1.4976470810625768, 0.11895090128151849, 0.8817473867652619, -1.0646263194717374, -0.12306667358622282, -2.0329383391482923, -2.0478051845963554, -0.5685230265908605, 0.7600263109667952, 0.5662328077132663, 0.5512046636426796, -0.6174141644312955, -1.4616434778891372, 0.2294714638999378, -2.28016128601594, 0.913554179429105, -2.517994806985082, -1.1805230576049976, -1.1234616018921013, -1.2469118357009203, 0.836876814793948, -4.338229861188179, -0.9515019718877463, -2.6315900024700443],
[0.2255742122524791, -0.020077193887451182, -0.5414604890971371, -0.2544782440258654, 0.07325478781255626, -0.179225946700166, 0.004910035039568767, 0.0740968494423371, -0.7062118946663455, -0.3704531217746758, -0.5123208817074285, -0.22186093775968302, -0.9197852530449463, 0.21553200692122668, -0.10807070960352115, -0.880666237371807, 0.14615382179798586, -0.04640289399199121, -0.06114132412721534, 0.18037101601183617, -0.466916091235809, -1.2322458315869014, -2.4667806248914914, 0.8966965560049353, -1.059917937483361, -0.559702205699209, -1.2046164296430975, -0.5692003896934342, 0.13895780049229722, 1.6932040816447804, -0.6511487756803919, -0.7494958004154629, -0.8154670258152368, -0.3715596850824017, -0.8020162383859005, -2.4015667546202377, -1.9727300098759109, 0.9411371222748083, -3.2282949923508664, -1.2073682538799273, -1.1562545943775429, -2.126927511294435],
[-0.044796026053926956, 0.009924310609705639, -0.18551603736622066, -0.2696422956467974, 0.05555224605693256, -0.0002872042438605303, -0.12296658366316482, -0.0022513574501670337, 0.08250128252381149, -0.039750398524821914, -0.21479616613717223, -0.17270776340742666, -0.04338713122836821, -0.057818796468181104, 0.04689966216198445, -0.154722694753127, -0.0880726674917101, -0.2613017456008336, 0.059543590704253586, -0.11185331485596736, -0.24195911937353096, -0.19046747659808308, -0.14015694155578695, -0.17916920400793848, -0.37721772367917605, -0.23800654880882957, -0.12859987466930622, -0.08275778846024281, -0.6511186539746762, -0.5668895417130808, -0.4687618712462513, -0.046983455216154164, -0.4868305565567244, -0.47569765712350487, -0.6835546702664994, -1.451693093954526, -1.3720459233570517, -1.190170807760268, -2.332554153007719, -1.351550137421254, -1.1562353867855018, -1.5020330788286702],
[-0.03113309953297025, 0.09450013820856777, -0.1894047381339914, -0.025637408968862085, -0.14306766537297022, -0.029728690857090193, -0.05371660237689122, -0.20068794050969904, -0.03353367383183577, -0.33467239017794354, -0.22794564864725417, -0.03889489358738558, 0.09942027677611777, 0.1134903507586608, -0.1117494434090964, -0.1122662369960445, -0.022997775740790902, 0.0032811364354559656, -0.10266486216788098, -0.20237643636092423, -0.21791042745547876, -0.27534943166046333, -0.1763331893825538, -0.3143590010182206, -0.13304363820455678, -0.2470249285533783, -0.14621392588723328, -0.24184521145384683, -0.41768360290213563, -0.28412864977189917, -0.13229996733538096, -0.47351056693937016, -0.3791586842104884, -0.4631252078280587, -0.4804780093201728, -0.15265401925813832, -0.5618738707289646, -0.6321672284607233, -1.0461314662626053, -1.020498232593841, -0.749773983490553, -1.2770825762980356],
[0.09627926805018897, 0.04739879890561693, 0.03333066803048042, -0.19403801194821346, 0.06740807107962554, -0.01806174283518468, -0.02526427904616234, 0.046400111243546274, -0.046125518607974696, -0.21366227464264484, -0.0904074155907055, -0.2067738194481385, 0.1458143648956875, 0.021822987993485556, -0.24246163658018668, -0.20194031027980708, -0.23056631654839763, -0.26960485346534224, -0.21694585461414748, -0.3402604740333732, -0.23997179709821131, 0.032016361768448034, 0.10763490914326734, -0.09727669569794035, -0.055619790529815145, 0.05715866860331086, -0.07394122651491024, -0.16883163158716025, -0.058590675556791844, -0.3683702250912576, -0.35776487948097463, -0.25068527237568733, -0.19480757474221022, -0.37133257261112407, -0.28713273492850744, -0.37792793676123804, -0.5381923175147619, -0.515047600420168, -0.6538556334821274, -0.44446656612957686, -0.16138121059825128, -0.8736148830703038],
[4.43201204423836, 3.872760407389115, 1.3905125856833223, -4.50140220581348, 0.3262521408876601, 4.050298894008125, -0.7642365307277106, 1.459326761297915, 1.6203593353654695, 2.685688577182226, -2.36758397688553, -0.7897589304822472, 3.207846343478431, 0.13005614732880702, -0.22248997016847405, -0.5003091890026841, 0.6700152035293884, -2.939970639259565, -0.9463187066375908, 1.6977347484198833, 0.9162683334002002, 0.7259865692028644, 0.04205532558084049, -1.2901472500250968, -1.6964980209726088, -0.6578812362074846, 0.7146512382141669, 1.8814256155075477, -0.6472912656967169, -1.5715332235172252, 3.459030356467032, 0.9563211193278083, -1.4691239136344119, -0.7089152453781562, 0.005459684943730131, -2.9161360804299026, -3.6583927363161477, -1.70601001695616, -2.4234914905319425, -2.7084563870855556, -5.459265521512621, -2.2713683823035016],
[0.22835802058792545, -0.16481188312286954, 0.17288357885450353, -0.04733639178160357, 0.11182009247126375, 0.09168543364538749, -0.13151734017050223, 0.10258268252697758, -0.06194701581661848, 0.060795690864670356, -0.2507985353121904, 0.09717521515570707, -0.12329895319089489, -0.14925554613348818, -0.0648354486419391, -0.09981555016449485, 0.004593246169375249, -0.13537086157573017, -0.1406217305016427, -0.14662614550019182, -0.30987002473791203, -0.2892941268127957, -0.4182047737517006, -0.26076983035473383, -0.4214166290160867, -0.3135438117363587, -0.23053172697589377, -0.24080552733442853, -0.6496255904334597, -0.2759302423796438, -0.5019706250511845, -0.2665390890118757, -0.4156242809607145, -0.3956697117640668, -0.39860746522626084, -1.82838624603938, -1.406141604439724, -0.8755537500318732, -2.22272415938358, -1.0167760584623409, -1.0567376397141715, -0.8378480296667402],
[-0.25031978446729586, -0.1470788982936908, -0.04526612144567025, 0.2657443489856202, -0.0460123798716327, 0.3784515780053047, 0.0684053921138015, -0.22512739857898298, -0.34503179053535954, -0.3329770548065494, 0.03206441562529449, -0.009021692964670243, 0.1471724081360272, 0.08613124771594406, -0.4270200225790368, -0.029024261904359284, 0.07634074014697922, -0.03381512356392536, 0.0886046655600642, 0.07650788371658124, -0.008635075543788881, -0.2347449345873501, -0.2108539435929854, -0.19472157524263162, -0.13756540445211696, -0.26107377217011163, -0.2326821425964733, -0.1302205347112289, -0.17530375567372442, 0.10981520069261493, -0.13324507078459635, -0.39894790439658956, -0.286951518036446, -0.705438768859475, -0.7756675462155093, -0.2176702449475917, -0.8513327441010001, -0.5270729199274996, -1.364291166834891, -1.8250210465154142, -1.4174871605575508, -1.2981679820143253],
[0.04325725649330572, 0.15192565467980115, -0.047460622530954594, -0.22715653740366884, -0.5024954306550705, -0.033465632679977236, -0.04787937749488742, 0.23677687074920353, -0.02786848028526195, 0.19256623999329112, -0.08761555294919024, -0.43353013072644603, -0.17135789452948375, 0.40232165677194526, -0.013335375431500304, -0.27185491748448287, -0.11300802531096371, -0.24211456674628135, -0.2782731241002493, -0.303469152140997, -0.5875470080183944, -0.2283336291526762, -0.38067698909809194, -0.38680358827938294, -0.503338800723988, -0.1185612870539225, -0.5971344606435836, 0.09897236622940778, -0.5843910232542069, -0.7784442887573858, -0.6453667013262513, 0.1033571829852469, -0.19846843971902192, -0.8228979891461278, 0.5905610463772739, -1.877764337305312, -0.8446743862450933, -1.479100051214726, -1.8809569339756043, -0.9334476027443971, -1.1423197555785531, -0.3448916431869837],
[0.15221277684449608, 0.09172399076492711, -0.14453200452329856, -0.029919713181637327, 0.03572448291009997, -0.16182338414356034, 0.18596298861262558, -0.19564587094105163, -0.05691408074507676, -0.3771385451758428, -0.15256068672160522, -0.2535003842790266, -0.296390192560267, -0.1987045652763141, -0.33538199604875185, -0.41719425088026346, -0.08396211475352983, -0.38323725516526086, -0.10810095469850851, -0.24334855157803487, -0.17795350384007383, -0.2417643261211948, -0.36171335535005394, -0.3080639738890103, -0.44209000183166103, -0.5358906817313407, -0.368375189319856, -0.05858898313730844, -0.6992337058031409, -0.8052300356511566, -0.7307321720366833, -0.15441096725630463, -0.24987913852689383, -0.5359425639310083, -0.32177247294236666, -2.0849155128623083, -1.4243944949572802, -1.252888406180895, -1.8488964638827268, -1.5772228320134902, -1.2824175847036952, -1.388129892136491],
[0.2794805677807916, -0.06979672710543858, -0.001929333614686234, -0.14369223268328418, -0.36709089961099517, -0.16363930663421616, -0.054101084755542345, 0.0892254962774701, 0.06454972739544267, 0.11148039450178455, -0.1582355059912913, -0.19729651374110346, 0.09867390630508403, -0.31178915891188663, 0.06295690051042464, -0.17098627286569276, 0.0045545737201504195, -0.08055938163999439, -0.08395241247911289, -0.019924271697143205, -0.09453714454011257, -0.17696090727732008, -0.31311618569569866, -0.17469829807572845, -0.3047321021959367, -0.12398084233596667, -0.08626949832767054, -0.14303766489889339, -0.6300036734548713, -0.4464969489252495, -0.4485985339798992, 0.10708638113949731, -0.22032522680055153, -0.20472400985562034, -0.21762058894984754, -1.4154681048165119, -0.8811205936485127, -0.14963171911372006, -1.791281121106684, -0.36141995813115135, -0.43228444021109247, 0.03562340497161553],
[-0.06279842281289856, 0.13725622895057665, -0.09705038755202243, -0.35191119456554243, 0.09158563260394875, 0.13098195586121328, 0.21853374732576708, -0.14698421927238345, -0.1771279630239638, -0.07335904887923146, -0.6142642919864968, -0.24520976431442593, 0.07063671150610508, 0.04422864278708013, -0.05832833892216725, -0.22046240601768213, -0.01631414332175508, -0.7676176682339885, -0.25119575805979455, -0.14260621578617041, -0.19957661288995296, -0.39232215579344343, -0.4470317708161652, 0.059020264372071495, -0.6133907635084273, -0.034339014731794494, -0.2386402330286388, -0.4377625365604573, -0.4551875970490879, -0.9294657615767824, -0.25215887240955437, -0.057056267703146486, -0.36949090718511185, -0.7758586753011096, -0.5945590191148317, -1.3541244730662303, -1.364199903014491, -1.1861027370918118, -2.02920852697027, -1.3940373283296505, -1.1715489005612578, -1.5596968546528294],
[-0.4708866002006292, -0.260741068769535, 0.6006272600472902, 0.6652555465956622, 0.4431987124747919, -0.08483105958518483, 0.006257357957053432, -0.20722261984067147, -0.16243163136028416, 0.35061014495358095, 0.20674585252930489, -0.005848103334025453, -0.029830141717500737, 0.11503956822410771, -0.40565720060022925, -0.48997773204026107, 0.195609967570652, 0.18407728306709972, -0.05619034860011251, -0.09505016534240628, -0.17756999930299197, -0.14456603592713746, -0.3013281932805653, -0.04715495319255363, -0.32224389700735934, -0.10260225122833376, -0.49557707903294873, -0.19119085587301568, -0.730360337851585, -0.265197840870072, -0.4230209611611019, -0.41010137096787447, -0.5956853681116004, -0.14782267149156292, -0.7873529804224514, -2.567137070061364, -2.275976851418611, -2.591679933856033, -2.8014022601124755, -2.741653286578656, -2.469801449370795, -2.6382959211156254],
[-0.04484095281408317, 0.13239572976100353, -0.04653609841042081, -0.13867868993301705, -0.08247779185760734, -0.08510005512920657, 0.09485958254345052, 0.007008865663119327, -0.22433721277651084, -0.053607825221262194, -0.1385546123453659, 0.16231050980899492, -0.19367604945139544, -0.18850200441623924, -0.08065718989994793, -0.29781317288472675, -0.42248720754247876, -0.28359333379556295, -0.15908783235763777, 0.05981617834733024, -0.30027376355452046, -0.4192446701094187, -0.40709910352106304, -0.1913545650738321, -0.7405609169678377, -0.2594098600392031, -0.4473953352554768, -0.3563730152215907, -0.6683199619738592, -0.22968134982512228, -0.8922377797374831, -0.2832006614298049, -0.3713747770032319, -0.08002414574628576, -0.49185919843772824, -1.525590702000648, -0.934244407774509, -1.0401571519300923, -1.6011884015060833, -1.1020826087020197, -0.8950785664834331, -1.092476592935047],
[-0.12229337978943118, 0.4583161876344496, 0.5800341970052828, -0.13599680848080795, 0.49747340380401717, -0.540097528927475, 0.03870921638736499, -0.05774030464957453, 0.04351595205814208, 0.13904155794328962, -0.024436617344585465, 0.09404122616478373, -0.4165655099213396, -0.12388001890091475, 0.16945065867728226, -0.05676195065081224, 0.18695498619682568, -0.5690443695905398, 0.046535304269124035, -0.022283781967596235, -0.14531074110177733, 0.06414014918655472, -0.16662804088010003, -0.48214207637304796, -0.9321298167437955, 0.23055762122375736, -0.6611316744262886, -0.3040546231900157, -0.8972137054251715, -0.4287579455850442, -0.8121530199638375, -0.3344966363444477, -0.2706925550865991, -0.37159245234876653, -0.21324012420650923, -1.54119284954806, -1.1415138018809432, -2.0539329333404424, -2.3583060309435004, -1.373754464031945, -1.3570999014242848, -1.5246787165855804],
[0.2291283830806966, -0.27431127281643336, -0.38090452756647575, -0.015017158532495582, 0.32672454876025936, 0.1657829605098255, 0.25987299589980517, 0.1762074196759346, -0.31159070706265357, -0.5940279109199498, -0.34324547622480633, 0.33046992758057764, 0.031051675280967105, -0.13811194895018497, -0.07486743691708628, -0.18791201699095816, -0.26105281522558, -0.056084195182660404, 0.15240340678254044, -0.2344351014060621, 0.03502761695491935, -0.023087407578308967, -0.2900176753947214, -0.5668546397310191, -0.30384521044481905, -0.24608701199209487, -0.27994375528998966, -0.3249033594481029, -0.25197623622338644, -0.8562653112156217, -0.3210673679910855, -0.3705159381363003, -0.672606326363942, -0.9461881998206454, -0.7238664892359571, -2.392762701721054, -1.8182440806168858, -2.0232373330932196, -2.122547150427178, -2.193118586787918, -1.90245896689268, -2.4517753859403117],
[-0.0015206403251002637, -0.37437595170758703, -0.6320300801923696, 0.06841915196506343, -0.1050099972992359, 0.41542963935758376, 0.307744615015451, 0.5440594564573619, -0.29199217697085667, -0.6258049333573349, 0.1687885478144928, -0.06884740264778695, 0.11282559551263291, 0.40067835036332156, -0.29551356360956504, -0.4443553261771868, -0.2606807680490878, 0.6058568172397806, -0.346870577411952, -0.13529396942311156, -0.016655693907329874, -0.5723746686873009, -1.28514214697042, -0.09342753404790553, -0.36751056681707933, -0.48287410165004774, -0.37895774691488926, -0.23124230503426357, 0.03928105643768641, 0.09396968085770901, 0.8019052754553939, -0.190068124775844, -0.37283507617213646, -0.7716122588388306, -0.7127588545811485, -1.0231874181685592, -0.9121951914121981, -0.21435925960149016, -0.9055941962354647, -1.5807593373616535, -1.4019486461227828, -1.6033345448991816],
[-0.2060145868558603, 0.2980061843404959, 0.1156945102372975, -0.06149811701604343, -0.09596661791985998, -0.23856395179460194, 0.005151933325848117, -0.11620098306110582, 0.09287962020782957, -0.01567336838000339, -0.05877116060000168, -0.21857856054979122, -0.038969784743368395, 0.17757796449433214, 0.04385842001625927, -0.18977979585546464, -0.4230894964052294, -0.3835408284678424, -0.25398179879017047, -0.08322090527146447, -0.12131798676068892, -0.2691475659493673, 0.14193642595439881, 0.09460266359266754, -0.5886217158226245, -0.2927028109297755, -0.557050000631455, 0.022689436234590164, -0.619894263939947, -0.10362879513645314, -0.2817529216300446, 0.14136918431875822, -0.4297473012197432, -0.2316837485243169, 0.20615449804606226, -0.753442584640194, 0.24597510553192908, -0.080253848937049, -0.7195301108502058, -0.0903952892068489, -0.8054140687101347, -0.5867758151062857],
[0.15902242406021955, -0.20355937070870345, 0.2509872184525435, 0.45969574114692247, 0.2927085924835664, 0.09081686432746296, 0.19019798348603417, 0.18307432891509928, -0.06387542185955127, 0.07522726463056316, 0.4938258788293778, 0.08907739314136046, 0.36663308894769364, 0.4238918052898606, 0.34122114829259703, 0.1652977324738714, 1.0213666284432876, 1.0323285416134016, 0.11350565682533045, 0.2776588646829625, 0.46565377605903374, -0.11853613086951645, -0.807813038020975, 1.0333072774252914, 0.7340478745933652, 0.33075653962597734, 0.33591008561211894, 0.30771451012120066, 0.784969735660103, 1.1270691538869302, 1.2344548273915614, 0.09094545579490988, -0.1430751373263449, -0.023968075407759918, -0.13843900808641255, -0.15932986075503883, 0.0906338447055092, 1.4427647823730503, -0.9504276458059855, -0.9767803181940892, -0.176517354520675, -0.27787685655966715],
[-0.044535114480093986, -0.12225128632993841, 0.06088937067063211, -0.2114451253337925, 0.1405712363273972, -0.06364683901663605, -0.25038578702089587, -0.1946343158023113, -0.3219053898400121, -0.05702858008266488, -0.23167281464211842, 0.02300533569323265, 0.2529517485519827, -0.4566959329543134, 0.13181157943455285, -0.4730620756194149, 0.44108708090395765, -0.03759405033460269, 0.0703092892479691, -0.3232590512365715, 0.6769127637755629, -0.17275275524639852, -0.5512947192376033, 0.02557434876553949, -0.27110281226315364, -0.6637499466667217, -0.2073892809807123, 0.0007881952886723994, -0.55323401833617, -0.6271079302718797, -0.11433275992332978, -0.20160061399887647, -0.9260040843417108, -0.5046322919765908, -0.5726538126120064, -1.3159857562225001, -1.1807489547827281, -1.5756259765671456, -1.669180206036353, -1.176900887273488, -1.5771791239559363, -1.363609273983212],
[-0.4453239874328778, 0.07705949650179407, 0.2559823544880304, 0.10102767996290365, -0.09889106813004114, -0.021762309428801436, 0.29080288103637686, -0.3856737765909369, 0.2819486262470345, -0.1524700491227143, -0.03470288804687241, -0.040870111737568986, -0.06833670090437644, 0.2852063908900234, -0.2537181169900987, 0.19531210129617121, -0.16436844615392662, -0.22064238639918235, -0.30625678835017894, -0.3304106533622743, -0.05682428930091383, -0.3472791525187002, 0.0050607862310876476, -0.2445732577007172, -0.32668844441178635, -0.21549962821403218, -0.585915912693358, -0.5235715291272139, -0.4279666003807949, -0.34367999724153964, -0.2314295087043121, -0.5201189727822978, -0.10221199733623483, -0.6246233785814024, -0.7821589305737986, -1.8845065661526275, -1.6472470288282672, -2.2348079278076387, -1.8181490248444148, -2.0622541673245056, -1.932381018042572, -2.2614905626893673],
[0.04065753705058766, -0.0015393854162503672, 0.08339077506542365, -0.017583054285323193, -0.07542923521290575, -0.2452700044605009, 0.028076600868154198, -0.01684443433558638, -0.09803847411467503, 0.055411260757076156, -0.028808643265423522, -0.2645610119384447, -0.026259694605770916, -0.11030310189945917, -0.1773198482826913, -0.1347497849532053, 0.007302531602776921, -0.2026971560311683, -0.20456665321852946, -0.25218326288091364, -0.14588761090541016, -0.1494445992853012, -0.16200008549268594, -0.10874572900405344, -0.44071221666665783, -0.31080043023836323, -0.4348764029837848, -0.12447811581895048, -0.7295220232473988, -0.4241979545305413, -0.5168918789461008, -0.32589131929216447, -0.3708650312287217, -0.4050519449594287, -0.5930988004394749, -2.0491680091983095, -1.7730476330284748, -2.0723033191398, -2.4659592425712056, -1.9380079003469897, -1.8585271878067773, -1.973185017489535],
[-0.12421804373603298, -0.42694517766501483, 0.09657588319707659, 0.20681041801233488, 0.015999466242897543, 0.0038118263912345117, 0.05071175434263809, -0.08588301190089329, -0.33038642644560584, 0.01494900036297929, 0.07079820771501846, -0.02868412283940694, 0.018644963248051257, -0.11634558701510665, -0.06385385187218574, -0.15807831687690074, 0.035061671591661715, 0.010765529483389552, 0.08867564717787264, -0.2717658256204067, -0.1880577592830178, -0.17363195451767996, -0.3064368399118479, -0.14027929171896927, -0.27128963079166585, -0.17016309910357338, -0.17836126535555047, -0.2517144640106817, -0.5522330062386233, -0.07540304703831736, -0.4475285041110648, -0.539391155005295, -0.47573468757124676, -0.4271373563171222, -0.758800872925746, -1.229373992295898, -1.1689746738750713, -1.1998198065637316, -1.936284316080164, -1.6569333439738267, -1.4809371737080068, -1.4442012810136808],
[-0.08104424402343917, -0.22422001066297986, -0.0004284300984471623, -0.23269102085188612, 0.09472185921121859, -0.16089433781279985, 0.02798928015329115, -0.017008644730366115, -0.2701168913950732, -0.2264800890730066, -0.223226740307587, -0.0417787495786379, -0.472910125570414, -0.2529167798424164, -0.24373099754954208, -0.2400546603490825, -0.4872670896357072, -0.8127593763254822, -0.05504565096880614, -0.7648803183850876, -0.273146257572683, -0.1310922423812387, -0.2693030934063554, -0.4475926921268047, -0.46007553288867925, -0.2993038180991082, -0.7092566342438634, -0.33735210903976987, -0.44437027670768225, -0.8564356308570014, -0.8124517232219145, 0.03756106279549303, -0.6028828905832451, -0.8309919492738699, -0.6671688180597155, -1.5924122702135477, -1.1804480628250387, -1.1277406664905918, -2.0027393412644314, -1.013682871690066, -1.1678379433924855, -1.542638677183466],
[0.0619340127217027, 0.21335610521204137, -0.2650501858269215, 0.5752464619582432, 0.27704183701649676, 0.3590327167907916, -0.17337123624373602, 0.10910683591711857, -0.032605985815612554, -0.5532102219210878, 0.44359652214823214, 0.003895389381684751, -0.08993441268938039, -0.23948131649531337, -0.19843180748418116, 0.16766518076552892, -0.5555900239381099, 0.3148562857292429, -0.15833656850279262, -0.14699425381582695, -0.030909557469972895, -0.24213100105332372, -0.3569151618222619, -0.49009933910393166, 0.07389653487487678, -0.45684479892069274, -0.03477980304115765, -0.2791833308112009, -0.77006938659484, -0.5272890377664234, -0.7172855160471858, -0.7379130987799881, -0.574638092481084, 0.1954537168034736, -0.7105319493955389, -2.3661719338075047, -2.1284754893252265, -2.5702053055975966, -2.6190737964984754, -2.586097520548482, -2.133492387890236, -2.2268167380941972],
[0.03307758171426434, -0.055687025143733575, 0.222882468664032, -0.11366866320036924, 0.14117901093391974, -0.14643757999937287, -0.05374539652724058, -0.2643501406572116, -0.10966126977988773, -0.004089378584574096, -0.1553307960536335, 0.029976019903584884, -0.20477453544366322, -0.0906887632493771, -0.2467796929995653, -0.4531610929507082, -0.10022124047359277, -0.23324806077832713, -0.11464107083293634, -0.36248503695678064, -0.24994224153498354, -0.14866820593909075, -0.23213085547247528, -0.15146063952539968, -0.46783090862374754, -0.14293868147586866, -0.1753314165636496, -0.07609952268413185, -0.4926974474640503, -0.7182048123183555, -0.5742524655250657, -0.2555200334903114, -0.46762647887334413, -0.7299867066241962, -0.46591862673802165, -1.4872572978339427, -1.3484104701583879, -1.4290644912906953, -1.8761387511212466, -1.5633171228115015, -1.3746088515488377, -1.4629083706228088],
[0.15152512699334017, -0.11196763668392, -0.1702778828217774, 0.27803621012726865, -0.19469097760252957, 0.4828059731649251, -0.34148555590231267, 0.059266051035630554, -0.17666100917705, 0.0652988898641973, -0.19522237511722204, 0.16615448608704603, 0.434419210114441, -0.32585517472265346, -0.29826356245541424, -0.08899943265460179, -0.1623752972130764, -0.075172587471546, -0.2980577383463396, -0.05490414112528661, -0.19037413836001676, -0.2547612084097222, -0.3220579885855687, -0.06483111503459209, -0.13863455588910822, -0.5037385796279801, -0.09454873542718839, 0.07729216594268189, -0.3345804341070537, -0.3816630530230242, -0.14297273515202763, -0.49797430563649875, -0.5882222671960797, -0.7056406449477973, -0.7721566442668784, -1.3737815649610403, -1.276303205004625, -1.4305147626613255, -1.356174795439517, -1.6939674557687903, -1.6538630051447922, -1.057484138535167],
[-0.08989949566808436, -0.14175573296685895, -0.40009552338572535, -0.0327867039548957, 0.32246857116170696, -0.15717246996420664, -0.17060102728007784, -0.13469921267203658, -0.373411737096145, -0.3721758700660867, -0.3111642690938264, 0.20602246994282433, -0.38880717552716276, -0.6259918470404913, -0.29001703174601073, -0.5760356716926065, -0.3112334327012653, -0.33219185710123167, -0.1011140798720295, -0.38568951056360307, -0.22270844962160577, -0.3742463218906451, -0.36472215067717006, -0.2174638132677086, -0.4766576013023753, 0.5246275874883052, -0.6141685758888151, -0.05595536101910131, -0.5097425632213991, -0.7779983956814038, -0.20571325969165133, -0.585215337926522, 0.09023699465228889, -0.44193548093678936, 0.11409879540842763, -1.1984366061572973, -0.8878131034826394, -0.87947833604108, -1.4089168921591972, -0.6852371648855353, -1.0648875300161291, -0.534128893098299],
[0.27541353824564335, -0.15240081111380263, 0.13527669854243365, 0.27911294175558105, 0.0560633600511709, 0.011909525047524041, 0.2950515185598472, 0.20271551761122714, 0.006035244873290496, -0.15383214487082372, -0.13952229498602578, 0.14882466852176118, -0.17398186484426453, 0.07555980894611367, 0.16954536308423782, -0.0010089678372416206, -0.4091375174053615, -0.4370874527345977, 0.12689985162563225, -0.10392834249835697, -0.17038976726481134, -0.2701786206674827, -0.2178467139200442, -0.2560521518838065, -0.514620288243075, -0.16891607257695296, -0.5324588603895489, -0.4396478834713344, -0.5690330255707842, -0.25378044951386364, -0.4738940076562994, -0.422707585300076, -0.3155911914122251, -0.41007336815191886, -0.8380187078935247, -2.093750271075291, -1.5732874872345308, -1.6635913777850575, -1.9761213143714225, -1.7330375001596856, -1.4457321824964449, -1.844771803229224],
[-0.025281247840876323, 0.28541539086274514, 0.21571262849672654, -0.23299897717858578, -0.15735246154995525, 0.11377384405131315, -0.09226628326127843, 0.06120090341853266, -0.09074265096834055, -0.0011221486795657736, -0.3134910920062955, -0.02503275243646967, -0.07287270826031332, 0.015749958884608495, -0.16869394088160658, -0.37969202163355525, 0.25598848896113446, -0.3744556286923753, -0.3152231112132089, -0.2190266411431895, -0.2164619300206101, -0.3583044490573632, -0.31373049603596104, 0.025695000983311043, -0.44443014915701, -0.19780758128202536, -0.3427226175334938, -0.21944248603973077, -0.7981797158368386, -0.8708536067026651, -0.39255046299617785, -0.282930778353639, -0.5809013244655892, -0.4556469525249787, -0.4770656894838596, -2.0035786519177172, -1.5713336878136899, -1.808994285822909, -2.1649029702597917, -1.5722809444898174, -1.6433147503220988, -1.8117330353457508],
[0.18542397069677138, 0.18091433239621799, -0.28282054519630073, 0.07071159480405936, -0.08637689498215262, 0.14201229725549172, -0.0312965367466732, 0.1380115437345596, 0.09887946458582851, -0.43584238468324016, -0.19976509339590912, -0.2623837139912022, 0.09916380112535364, 0.16769274981052476, -0.34218988469142153, -0.3156070650423583, -0.6678620673898777, 0.07578246544371106, -0.061331240908784446, -0.026008272229236712, -0.2143169393214645, 0.057581148049886476, -0.22148883886788065, -0.7812380844695954, -0.00852429953603385, -0.28511412917573126, -0.08732768148662941, 0.010858811662225151, 0.0044267959333404465, -0.8874443941439114, 0.02530501535924944, 0.3389766148836113, -0.41362044586227636, -0.2587126522798882, -0.22303111371483866, -0.5947178982873078, 0.607726119613658, 0.12079185731957728, -0.2706239488528071, -0.7207481221048069, -0.26883521089630297, -0.768176380431922],
[0.0979545844791109, 0.04521885520447128, 0.22514125203995972, 0.28699642312270207, -0.43005068688844933, 0.03851074075968362, -0.06811889513653772, 0.038032344465820184, -0.35112578462778304, 0.02858328122654128, -0.012412465099384217, -0.37241563599841127, -0.21424602307662738, 0.04546533669928419, -0.17053341191711, -0.4598007167920965, -0.38637639885948055, -0.2988658331591865, -0.570930604820554, -0.06673991874167166, -0.5033701355481416, -0.41050374453238203, -0.4997459688188688, -0.3098007258481459, -0.323284957005129, -0.926163668706666, -0.637343729785525, -0.04966810837771013, -0.7330974273306133, -0.6832244493090468, -0.8299847500441445, 0.13755533902652456, -0.2536941835311381, -0.46931141773525165, -0.6240551630687925, -2.3855366738465738, -1.8410263384219514, -1.4593721186604398, -2.32032757183691, -1.6432392699962246, -1.6432646896990513, -1.429705462179859],
[-0.005103102384573893, 0.5700367446260858, -0.40750564800630756, -0.013712085843816503, 0.11168197618211619, 0.2250002265714254, -0.6243456690524749, 0.0011688191551525507, 0.14393083801954146, -0.2834949645224614, 0.11703521870839348, -0.1081904726791435, 0.2592459561582379, -0.5300623593314356, -0.02724250147352231, -0.41917210902576507, -0.17827172199995595, -0.11285759384271045, -0.6433188533020716, 0.19238300431814312, -0.03125908916624345, -0.079289024372477, -0.4185804416011369, -0.2644900404413494, -0.11103046979659312, -0.6610744873622608, -0.03436258997059978, 0.1119027595737856, -0.6437067023652467, -0.9222498440709541, -0.38015070497293474, -0.3337202688992591, -0.505138629444197, -0.4303526558132694, -0.801106731828042, -1.553526359489598, -1.5870260895200714, -1.6604331357461506, -1.670627181523233, -1.2379415228009591, -1.56421695492957, -0.5515132635305975],
[-0.0641283001487898, -0.04379939233210839, 0.26962180351650417, -0.12453581810518675, 0.12140217682942621, -0.022733402873785016, -0.03970045459949424, 0.1592457124544846, -0.12377425020624072, -0.06816192139448327, -0.008769174202015072, 0.0014580895978170763, -0.13461409559437806, -0.019715421669342516, -0.060700427301258485, -0.16773434895514533, 0.04540450094482436, -0.05625363860609316, -0.20093154379264258, -0.27864819609681063, -0.22493628392785156, -0.27410058153559386, -0.212031009367969, -0.25337822892769807, -0.262787090164654, -0.29051939392379256, -0.2302544177826726, -0.22062136033173693, -0.7569236384366824, -0.37959018913759646, -0.5006353560001162, -0.311371811266719, -0.5721931159847611, -0.4841699157868688, -0.7089695008438082, -2.459978097366283, -2.162808047756063, -2.275023748797365, -2.7392381008498328, -2.4744254644452846, -2.197624083281781, -2.299955340692202],
[-0.5419468233813419, 0.03560024753930388, 0.36280406125105724, -0.19022771880330133, 0.06029321550935391, 1.6492215760738216, -0.13759661942946638, -0.48948067463394307, 0.059349398026310646, -0.111095844555362, -0.09551591168173038, 0.2672637787852374, 0.6250906551673986, -0.36290851361051, -0.2947903668503934, 0.3343040293776232, 0.07881804315680786, -0.1140858548768551, -0.10112742054941543, -0.5950797120985303, 0.0726514759611617, -0.15025985508245362, -0.22644858080760757, -0.28666913429603386, 0.19023417822214678, 0.008144091443216905, -0.8617696512631711, -0.44257083512199696, -0.8478294166382373, -0.6686292456950528, -0.22700813915844664, 0.2310126087117033, -0.701386476707745, -1.2008662162464103, -0.8797080064836854, -4.081018075780308, -3.77826472538445, -4.126673694039032, -4.101939183552342, -4.221215416342805, -3.990851495149376, -4.225425623102951],
[0.05163891412228341, 0.020362004809444237, -0.14121695773113505, 0.16674750290154722, -0.23288359638978479, -0.03512818596041573, -0.11080809390299819, 0.08227193173996931, -0.1907920148771567, -0.1231610411429964, -0.31706427387929353, -0.37931846485432485, -0.3786482097517424, -0.044092707310771445, -0.07832414802208305, -0.24282147040686958, -0.5039854895388808, -0.09108530575001376, -0.6777014267886854, -0.20822946267624382, -0.33924864185042014, -0.44900872129321545, -0.8704007554739419, -0.09364419165992766, -0.44142703555901147, -0.9037469605211079, -0.7351706319653215, -0.24616275802342022, -0.6939654411764054, -0.4515919044517871, -0.4558704778588767, -0.05046158215321646, -0.5684158402791137, -1.0552326303229778, -0.6830362040016132, -2.4957801724795114, -2.112332737723846, -2.201232086445909, -2.8789441327414176, -2.4118177147462654, -2.0063455201577187, -2.4177055128268883],
[-0.05244206144988943, 0.3639237762016029, -0.5612174467554195, 0.0680705336958913, 0.498313185590247, 0.07051929772714244, -0.35218297869879645, -0.28513586591202594, -0.002272601560086151, -0.6546399888487789, -0.09606111469404936, 0.23743710177216346, -0.009354259243331962, -0.29744178717211667, -0.2904133840294038, -0.19998547305274655, -0.14246126264088607, -0.43099640413687856, 0.36709998823593415, -0.23192427738103705, -0.34590998346981716, 0.2788541506940645, -0.20639410950452375, -0.6627150321682735, 0.10557366587717211, 1.2007571154911068, -0.26237994921205515, -0.10656832878655956, 0.47506879092534254, -0.8573478716707229, 0.17097362964889853, -0.046590889106888134, 1.4289563166591734, -0.706336365891982, 0.16327429750775235, -0.9577227143118039, -0.5886142582128943, -1.0913981190662625, -0.4714441146321721, 0.11712885523511339, -0.9356075710776492, -0.19171237878007763],
[-0.06347276684054762, 0.08949165223597044, -0.2275751146173781, -0.20146323361530416, -0.12634298142348127, -0.26037440147941554, -0.16988123263133842, -0.05600111680579049, 0.13203684045180236, -0.20505413872548098, -0.4203579091518801, -0.046174538561395514, -0.26676567501369414, 0.1831779214615405, 0.06501328050631112, -0.4724883278740819, -0.6718172963882534, -0.8206727588030226, -0.6058036601451929, -0.28294148212824616, -0.45825833833387847, 0.32778412738706564, -0.25983444831843455, -1.1022148995545913, -1.2514769334571498, 1.0532512138477086, -0.9617424246767391, -0.09501779252595802, 0.5418719247665403, -0.7505874896372406, -1.0892954354660425, -0.715296480932692, 0.7414922639373336, -1.3475741288919578, 0.7273329817387261, -1.4383639167073865, 1.340885006230189, -1.478005224551601, -1.58286159164998, -0.18951491855012767, -0.7470980716799316, -0.9522316795271472],
[1.0059076511319467, 2.771098246577913, 1.347654700665691, 3.1548601216227756, 0.1029257534884707, -0.07589647651915167, -2.136325920452993, 0.130995267083375, 0.6855904041136095, 1.0407637351611547, 0.8593925608909899, -0.16389456084937715, -0.5079270169864702, -1.578577324296544, 0.012493142308455938, 0.6949216155377687, 0.32364873884612916, -0.19937682105937887, -0.0005716119829003155, -1.3117658594055102, -0.9431891955103526, 0.22019332295256824, -0.2211144604793013, -0.9361727052257632, -0.46844788924226133, -0.20910784664685722, 0.30136177227099925, 0.6273415856302392, -0.8912384074814245, -1.2401699765960583, -0.9451721694610834, -1.101569279144869, -0.4568641523721895, 0.7637141074546165, -0.4689634970560205, -2.795523613674721, -1.8578087725756103, -3.6019263937897765, -3.2486265733990507, -2.7669168361667227, -1.4814894703067867, -2.113464328411505],
[0.29842453028914223, 0.6108800063718448, -1.0095464936348866, 0.22678583522770443, 0.42518097735118143, 0.44166755893722975, 0.15270016112122964, -0.024092607481980968, 0.28524508044626384, -1.1691172783595747, 0.3698522660455645, -0.1435375213011618, -0.01080445102172722, 0.03269507191460693, -0.12305149447509009, 0.0763110171237426, -0.8370682604987928, 0.003983681369032407, -0.07325386409411333, 0.15606457648207486, 0.13774687157714127, -0.3436309581904915, -0.4876392341858417, -0.44168789679457277, -0.13899842715711322, -0.23783837093455798, -0.5684505216186676, 0.003156863891863693, -0.49164272480952204, -0.9937471701303584, 0.19056499440045296, -0.6524729644602179, -0.6336654197175263, -0.39140249739718963, -0.5784965050370562, -0.18516462923598387, -0.9657937399496509, -0.9363287515378568, -0.7903459918782285, -2.0668472239820743, -0.7703165138186494, -1.4956280210577695],
[-0.25084501883733656, -0.13428299514355024, 1.3798172865875988, 0.8053166176185145, -0.6393426168058947, 1.346693367528009, 0.16989201997334444, -0.044075956333161616, 0.37534021308717824, 0.681284778495868, -1.130502148218105, -0.8517658321411072, 1.0814346606384515, -0.04947628202256727, 0.21926047689216238, 0.7388265610690875, 1.0658979669016415, -1.4631342640775797, -1.0399275663902467, 0.04806656134118867, -0.9437177982412795, -0.7953148460299001, -1.1848349349156975, 0.353619290304854, -2.224912391909955, -0.5331485007597421, 0.10862458479764714, -1.4740239585696011, -0.7576589251238528, -0.43066262467578936, -0.4249324532595319, -2.373781887468775, 0.5237368132917858, -0.5934405144690094, -1.111848351830226, -2.2965229298636105, -1.6378968015685638, -1.0265749712562695, -3.3759084864480804, -1.9430662714386713, -1.4144717515978988, -1.6790127438676796],
[-0.24275214861780375, 0.1775265659029187, -1.7364240665902204, -0.8692961071118497, 0.5574911952144335, 0.721121731304543, 0.6160901667827305, 0.41212201284238154, 1.2242928866577714, -2.7490098825565004, -2.628437230248884, -1.3047972043085307, 1.3485323198707884, 0.49807337982199196, -1.3560538673339901, -0.16240783370776687, -2.9116443951750273, -0.8478993187570513, -1.1846481944734668, 0.6972498237229379, -0.11395828710897206, 0.5266594868147205, -0.30944756198236845, -3.926092075408135, -0.19501337421226037, -1.2061331179811672, -0.021401052329860004, -0.1583064053737175, -2.7159666693907165, -1.559545449589167, -0.6529893752518923, -0.39700618315570735, 4.88970724361615, 0.2663338659304826, -0.8926800380790059, -2.8515407545385307, -1.3415272750646632, 3.3976142208727333, 0.42491268519012926, -1.5250283959183084, -3.8701125605516125, -3.857496263378082],
[-0.11789734171859069, 0.07612352425453361, -0.36374261905907523, -0.19622944702509115, -0.14602604895735546, 0.17986707051849551, -0.004952404978778958, -0.27777943765409807, -0.18298429095601565, -0.3133197145252584, -0.11790129684579734, -0.14793501213419427, 0.16692144970474534, -0.22709329445048426, -0.10959966596723046, -0.16306519171634185, -0.29710516480068044, -0.2529528337171896, -0.11580773555128919, -0.10595856092323021, -0.4605177461804014, -0.32336326611650423, -0.22563743304811962, -0.4878711656927413, -0.4259375003851362, -0.37380369395362567, 0.015914801517830354, -0.31405300011913234, -0.502578703773376, -0.49591531122839755, -0.5116652987940045, -0.29907539216102474, -0.38140133907390844, -0.06779241748309156, -0.8913253534744595, -1.5775123366751802, -1.2609995459767323, -1.231262447885744, -1.5545080698369302, -1.548144906045741, -0.9329264894378333, -0.8937619474026581],
[-0.18896025500691876, 0.9637113158817134, -0.5770063608154511, -0.7132416454457696, 0.3851042189711807, 0.904136140700892, 1.0339514413951396, 0.4536901059919612, 0.4655192078298316, -0.7411059135749106, -0.5904562321039898, 0.2243031372297898, 1.0899354432545227, 0.9719793502086839, 0.2135915971948218, 0.08860381710814635, -0.1359527681954147, -2.3521061676690476, 1.1632559161875051, 0.5884110798633934, 0.8269961088134808, 0.6127310585711135, 1.0423358005169157, -0.7883707471852657, 0.63609588053237, -0.3003438169727123, -0.33087665814783845, 0.7628165119259416, -0.040483886350331316, -0.7239258383829557, -0.6948615536261057, 0.8146721230624737, -1.0975219835267127, -1.039207613398292, -0.401039233978144, -1.7801184982758393, -2.2967279247792254, -2.2414615162082834, -0.04142212492544008, -2.8278150481502693, -2.1053363972432937, -2.6947061465684143],
[-0.09534315019538676, 0.0616032432898827, 0.09679115954524474, 0.10874366714210183, -0.09303003047021413, 0.28921361982161375, -0.057876347553798474, -0.11220448186033806, -0.10319646572696566, -0.045258027290837465, 0.018307353210870698, -0.30935120681413963, 0.20225962312439968, -0.23888607724804206, -0.19420143762253406, -0.05691830694738315, -0.1503240413888319, 0.009116761037880738, -0.13169789197883838, -0.09290361862495605, -0.32202794147831293, -0.2405107478738603, -0.13341688664321588, -0.13569480730878944, -0.18896021800968588, -0.5949854076849108, -0.21034524095519794, -0.13666882031888697, -0.5891517951357929, -0.29910608505621744, -0.661391283256528, -0.0455009086072051, -0.17080956505066736, -0.5482206608127218, -0.5091709699443364, -1.0049415862119235, -1.0819464927688567, -0.7236486842779654, -2.024352073213513, -1.467438989152982, -1.2432865596408043, -0.14558502290415087],
[0.14564244735316864, -0.06391263584739748, 0.7466342592509632, -0.23436795806892854, -0.4905780331083747, 0.12752199143230897, 0.44799303023567855, -0.12208291253230645, -0.25517606072581406, 0.21435109359683682, 0.2684870210494337, -0.6406398151378117, 0.35233006726063915, 0.04970519776734124, -0.2808167876724065, -0.20891018876624035, -0.31002661919245805, -0.4504982686309066, -1.6682582226637286, 0.027622508220040396, -0.4050635353687753, -0.381777175161506, -0.8320413476993671, -0.41119753972873485, -0.7321733512441958, -1.8126923678387177, -0.3277223273262518, 0.25654237312987105, -0.4862611841113248, -1.3394336783799583, -1.128856808808956, -0.03757706264048641, -0.3831190417627904, -1.3216834636246202, -0.6161775102875318, -2.5362682908793723, -1.5220793772237968, -2.0008954737220304, -2.107976199862555, -1.1795146390622269, -2.106296100266439, -1.6373402042155238],
[0.11044146170065487, -0.1513466208704818, -0.13622503542530665, -0.20568903850021844, -0.045277130773524306, -0.056906699999383054, 0.06622551113306983, -0.05069694544620324, -0.09226602463997059, -0.1593309088721122, 0.040906895925539984, -0.31964247427149006, -0.19651715467135067, 0.1035895033907576, -0.08919532417812265, -0.31329555094149436, -0.048850612411980805, -0.2638693005695124, -0.40530861150662467, -0.17101056894899175, -0.15166979452315604, -0.33888130722213144, -0.36102822114291383, -0.26169991826796474, -0.4393875046051667, -0.5237256530562865, -0.51753488185081, -0.05060186168380284, -0.7003655750419349, -0.49203226250811966, -0.6192689565265815, -0.3971700419359921, -0.30372238072102614, -0.31487371795218366, -0.43148365411111217, -1.954782450305432, -1.4023780743088703, -1.5326622829525598, -1.7181615210339314, -1.5508356820574256, -1.338247175214445, -1.6904370727984168],
[0.2356163654750906, -0.10677094730610341, -0.1605402858485463, 0.054411196747682744, -0.07360681350353222, -0.2762085640065238, 0.11465303582634406, -0.11926437388613724, 0.07153644942086144, -0.03125533103227787, -0.290060609191387, -0.4165194071025215, -0.21150534584413921, -0.3355835581747929, -0.03824982330137762, -0.4587774197700744, 0.10249123670777864, -0.16011709063304008, -0.2175606834954503, -0.4731319723206961, -0.362986634610287, -0.44731538387942815, -0.3845024612163905, -0.6018681645389409, -0.42889372768733586, -0.37180180202801905, -0.3409689444604904, -0.545291837312111, -0.6554712374520716, -0.810752565106627, -0.3015577986801303, -0.04535080625862878, -0.327184046086716, -0.4177269845953426, -0.4749005007573548, -1.9743015043127585, -1.2276504881171912, -1.3867445599131119, -1.3386659058924668, -1.134016165117984, -1.0286823047997118, -1.4731736173910195],
[2.6906496607882353, 1.8193042696179125, -3.2364363610267306, 1.2088246710523447, 1.3534777001051905, 2.0846626190750968, -0.5232387131218904, 0.32446998327769466, -0.6659688973677248, -2.0123405668171572, 0.2556187240742031, 1.264450056345974, -0.5011086347434267, 0.2078968070130599, 0.6290699014051049, -0.8109777384630067, -2.2241348272041885, -0.6099247423970626, -0.20237112991000425, -0.34218546212181544, -0.15813100204288405, -0.11431958413620759, -0.4862948159126468, -2.96264760410449, 1.2404549598331476, -0.6542617313641872, -1.0624947094757773, 0.5359277210212824, -1.6978045916816953, -1.3310508886709744, 2.5096770121628498, 1.559878681997236, -1.2811601897348368, -2.605647534848308, -0.5272626781312625, -1.9560366778969678, -1.708935072575533, -1.0609077894062882, 0.8544290157138521, -4.447129786045963, -4.838273994036848, 0.532457959593841],
[-0.15523455600562208, 0.05515439119803075, -0.05565897183724994, 0.2899286101075993, -0.0376264842359167, 0.1521146854649293, 0.1751695925629301, 0.10889557439929616, 0.06134004573478471, -0.07116162244178667, 0.07957991622796412, -0.40702055886069927, -0.10463143126884487, -0.03434726599993597, 0.18928588077523456, 0.5042203627676395, 0.5915879070789182, 0.31361328725215826, -0.42123391122595377, -0.139015150469488, -0.1752950371876285, -0.3433171796997383, -0.2027762536245341, 0.41619366814687175, 0.07662067988119209, -0.7004582443074051, -0.15926683218199603, -0.07789383922197467, -0.13825900759108412, -0.28931081883237925, 0.5841671295485146, -0.684026184584835, -0.7702462386043023, -0.47106272713710273, -0.20740235141946822, 0.027491363132726435, -0.3817437371846575, 0.9024641568666835, -0.9033414312980528, -1.0336511844775158, -0.3275228955419372, -0.6460305678546461],
[-0.18528832672807152, 0.010931107099394329, -0.11390408980021002, -0.09757806783444846, -0.20421404021541756, -0.07612967292366216, -0.09514459264276116, -0.1631327967673665, -0.04484219309164239, -0.04885702314184551, -0.09253491991470793, -0.17515215501734904, -0.0745550426862924, -0.16611243333264308, -0.14031954556113646, -0.33508106165645685, 0.05413009153996616, 0.010387589594252167, -0.34190870861129913, -0.23754381359330842, -0.17331959934317315, -0.37067745360770826, -0.1621010199719529, -0.30634621869838724, -0.46952666701568535, -0.3668470733187472, -0.2948410622189349, -0.33048408341973096, -0.7632234995255619, -0.5584955711359743, -0.5407017806205734, -0.18659051555740142, -0.03712455968471734, -0.3420080153739146, -0.20446415467011955, -1.2093354415948767, -1.1822598550746377, -1.4399158124037879, -2.20393648124552, -1.0500675304210583, -1.200553829853409, -0.6841528296856072],
[-0.31383145019021436, 0.006636434022185283, 0.11551358277755575, 0.09372743461454967, -0.008900874237616176, 0.20350416739814875, 0.02835463033859774, 0.04831333830726752, -0.13108854051317634, -0.3140663759021594, 0.0019007207498960548, -0.09459266143217057, -0.19676347491747312, 0.22087673901332758, -0.4259855035324858, -0.24037821771246973, -0.3398000887535245, -0.17979345291248125, -0.43442346239397245, -0.017774848584025674, 0.10258048767091975, -0.3209829369824987, -0.358289448572903, -0.4677173110082573, -0.29887823316001166, -0.3120005087199366, -0.48388513021911367, -0.26812184926559135, -0.6962482175951287, -0.7214681446363296, -0.23313858136290425, -0.5213309830390601, -0.4008606196788179, -0.34276274369882404, -0.6883042229379489, -2.1070054967471714, -1.9390909410376345, -2.166391687323815, -2.0015128212981805, -2.3063113161505293, -2.078873484055969, -2.576610414917845],
[0.24858820027426395, -0.03229510792201797, 0.06613229309615408, -0.30610559835608975, -0.21026962168854926, -0.2041197467358179, 0.05001321694500142, 0.06400640337795876, -0.032860542666952465, 0.28069402979984126, 0.3246276696820772, -0.2833377425825732, -0.10262124821960546, 0.002122698185823188, -0.002797061721734869, -0.30106360892211165, -0.41210148688461046, -0.4615785812083123, -0.6042140423465692, -0.3757454615871456, -0.4979636863825631, -0.30905592289975414, -0.5425921964003789, -0.2171323933609643, -0.3962012083514524, -1.0132099317443821, -0.7437922323571791, -0.1375321768044332, -0.36380991910986904, -0.8081649119391555, -1.1575684767240386, 0.3038486729487907, 0.1656941182793304, -0.6324913251710647, -0.6372401149564834, -1.665989288710998, -0.4547058363667633, 0.923728637986628, -1.4443054589278395, 0.007866554139576555, 0.3135651480854393, -0.6558574666922355],
[0.03249225955260438, -0.13925947785540377, -0.02376110966724406, 0.23765071502528087, -0.026269093136634926, -0.07495581736236516, -0.02302453821297352, -0.12919403233778964, -0.18473433635344522, 0.1729147224084195, 0.07478613793882316, -0.225945288869976, 0.14971660436546408, -0.015383388884311587, 0.007461832652635514, -0.051965054234731835, 0.022490423481969026, -0.1206647528930857, -0.20511455941888623, 0.0004644266011643722, -0.09217137532673969, -0.3414084296530644, 0.3219225962895291, 0.030800169688195787, -0.3615142804746356, -0.313269210730024, -0.33738504288881843, -0.008105616304178897, -0.7974144096953095, 0.7522106280015013, -0.5627723679759616, -0.2230398339071028, -0.06995079612233673, -0.2158015614711698, -0.12302266476198144, -0.17103096076602414, 0.0026593005278012696, -0.6982018161423854, -1.591328043108975, -0.5109054197993966, -0.9873188576383184, -0.4143763508413579],
[-0.11355251163193637, 0.0015029100289423328, -0.2388108315775066, 0.07898559704433024, 0.16496890207264067, 0.3213873635303642, -0.29852797374800577, -0.33611889418051205, -0.03643852623136744, -0.2928391059637518, -0.05349311788133852, 0.08950357252437158, 0.18429814642166162, -0.2242761501647261, -0.20891669514408986, -0.050228848874951854, -0.3076794638785995, -0.24874678471431555, -0.03655040067929438, 0.1205525669809887, -0.14249008610652913, -0.14946080299638978, -0.18061513699250253, -0.24688114186885887, -0.21441763421304555, -0.1945926117133728, -0.3542148381900815, -0.12439896780694833, -0.6876327048499685, -0.4316199739265763, -0.38104692011130015, -0.3276539524841119, -0.5000034627206088, -0.522867901626725, -0.7988823886232814, -2.5169295595679584, -2.350110353484338, -2.6171130347321845, -2.6898296252296063, -2.924434982441562, -2.471312064377728, -2.7152486853916926],
[-0.24847305265273795, 0.5478395485254351, 0.36358306752659225, -0.3043429940182337, -0.16622686423912064, -0.3927778578337023, 0.0027082852293415045, -0.24916801464307098, 0.2222783162248225, 0.09564445542569779, -0.09694971228636819, -0.05049792202151647, -0.3124596507213723, -0.0830370381563456, -0.024060744266540705, -0.21113527127080947, -0.09473080164645087, -0.055441305853998944, -0.3368720141002664, -0.2697080843852913, -0.03471545401978586, -0.13309471155606453, -0.22958356172423422, -0.05367612458525776, -0.30436865271220653, -0.19971504382596678, -0.4933035428299972, -0.11741602782765602, -0.911749617864806, -0.3519013559137894, -0.693833013340517, 0.03321495322083265, -0.0036259919962161186, -0.31962664514444183, -0.26547497582584045, -0.9097810143826511, -1.134495094109836, -1.5957185052390268, -2.4779346445085513, -0.6561259468680436, -1.0745074209972563, -0.807201213081272],
[0.01731875581661007, -0.06696590926834153, -0.07087654284649192, -0.22661299598850043, -0.0744999659090124, 0.0471752070760666, 0.07670916156732528, -0.17754624347826475, -0.10536105576825812, -0.12398923796640195, -0.15415405267395027, -0.20457434771657554, -0.045525844219440774, 0.13612727340570774, -0.33419524971638354, -0.0451961656634441, -0.03039982227069568, -0.16080656302449994, 0.05918163120763884, -0.2022010123100778, -0.0629424096829026, -0.32317264757210196, -0.46352302178039356, 0.06899359927356043, -0.1667725711022996, -0.10022724829082724, -0.1543804074000384, -0.301349575735494, -0.03693797120814312, -0.3588552369839379, -0.2729308378157327, 0.0895242766059951, -0.501064061029016, -0.5539427966354489, -0.15829812588902364, -0.20915893106904165, -0.6345547312889654, -0.31877431953407426, -1.505853680212299, -0.5642785401268514, 0.09675787248007614, -0.8449034940489987],
[-0.017699116898225745, 0.0312643072696428, 0.1455426805377664, 0.059557483282143164, -0.19979814998577475, 0.30277451109642667, -0.025915854515717946, -0.04728156149205246, -0.0724623191038128, 0.06815087759973153, 0.08979953558684331, -0.31455434099473895, 0.1394983043292122, 0.1998617321044307, -0.19883422176137766, -0.17475572618163565, -0.27414251572989956, -0.30460885535880244, -0.17471468324599873, -0.056311247025551486, -0.3234464136864025, -0.016141837342946636, -0.3832244238598919, -0.09275464784715727, -0.04860791219877174, -0.5462504424964032, -0.3537177032972824, 0.09353051563293849, -0.4970680982446194, -0.38781781135421967, -0.4631874394261838, -0.09488580229826948, -0.04893107100311255, -0.5169114832027448, -0.06736176439966106, -0.7636666921621112, -0.6876744763502501, -0.4359422845203622, -1.1258735483616529, -0.4789884664540819, -0.3018215803892248, -0.22255336424481628],
[-1.3586475203429302, -1.1079601659488136, 10.792595912263, 2.981459413748912, -1.2010516707562306, 3.152082297560419, 1.4737156617562457, -0.9994713941813345, 0.12016654624027173, 4.256026490523886, 1.3711194354287168, -0.15540261706134215, 1.2963285760472518, 0.15282761263140465, -0.9060099669136128, 0.024701587034694078, 1.1148239192523217, -0.3214057839559279, 2.306061036004914, 0.5577766634123532, 0.6359275794352174, 0.6320005253371588, -3.0149269078179937, 0.3619705882707812, -0.17057551684423275, 1.1732159592513998, -0.6534218976967817, -0.40251902827885655, 2.220540145241101, 0.02620515091970794, -1.34731578538963, -1.1410476527151778, -0.3158997246212539, -0.8843914164199604, -1.1567614231819812, 0.41852714069695157, -1.7767472896808456, 1.142130563279474, -4.827413454262503, -0.5050661669186056, -0.37421478905460415, -3.3484952740052027],
[-0.2380804970420826, -0.271260240218873, 0.3378944381034563, 0.0764652736061355, 0.30225979525847085, -0.3097353415464556, -0.2163494925169001, -0.21812412359241873, -0.7071526324742256, -0.18137964403281928, -0.022223228829319534, -0.44398146234940344, -0.7278702441224908, -0.20667724571635337, -0.11667260897683358, 0.3523117262001174, -0.5081299250036938, 0.6271095679947986, -0.3486810623486977, 0.33975975846745793, -0.42374983276779304, -0.7893722370750395, -0.9616168941670528, -0.3172325921510674, -0.8929305082373258, -0.6838370216253667, -0.8595330513987366, -0.7840425606910768, -0.7269004552914192, 0.15391460456724568, -0.298025300924623, -0.4586548868400052, -0.6359710659051035, -0.11737938280699153, -0.5757706104770627, -2.5960575999482396, -1.9803294265646667, -2.7708902902472228, -2.355498329228038, -2.494482538203502, -2.201064554933422, -2.7265596652812953],
[-0.34164177577452065, -0.022725721953792542, 0.11940908261647089, -0.03515394297257349, -0.20275476185745994, -0.07451988359561562, -0.062430504770326904, -0.22164187548095526, 0.04557077099405021, -0.11732376080087425, -0.17239163177164835, -0.17395067139125703, -0.2248536571047881, -0.0899268022221761, -0.19902247947335183, -0.2550275025974175, 0.06544717757219898, -0.31018915364854016, -0.16560034477018332, -0.2041014554000725, -0.1863101508691696, -0.10783029038605517, -0.3044597290121325, -0.08785836952259178, -0.32784956176771113, -0.16419610301625578, -0.2365575676351224, -0.208678812963222, -0.734856151759429, -0.35384273544632255, -0.4661954308832794, -0.1297800787276563, -0.3800189230863828, -0.6037031114436487, -0.7406769453951036, -1.7646418424535646, -1.7778332684161742, -2.048311261065155, -2.7893726508065715, -2.1545557456574267, -1.889630677516147, -2.1999211936782177],
[0.43397117240350364, -0.2127775461812054, -1.9573535736660337, 2.10083655881794, -0.0756170936017615, 1.2159033547091596, -0.08241087216248143, 0.7662432395348846, -0.42486455297817394, -1.4743287963794913, 0.7271377513562187, -0.39628918965895965, 0.15070519352899872, 0.535115422991396, 0.1260761950474609, -0.03706268271218793, -1.130076693822622, 0.47890789464325034, -0.4524455762007744, 0.26147173000396556, -0.5355318751858057, -0.46267405855865473, -0.4673948500712868, -0.004913283607161064, -0.8712886426188884, -0.6936528672162479, -0.5922459261760702, -0.26928075706839855, 0.3414620205256563, -0.4341734883158882, 1.8597078309202435, -0.8363550053965054, -0.41094343913702136, -0.7243983506443835, -0.8580500150084049, -1.6728964668469488, -1.8794163923670373, -2.471424190570802, 0.7094953329110955, -3.019997044354981, -2.5822178198650825, -2.7181375125858622],
[-0.17741501629214254, -0.0010112966860356927, -0.14885985619481157, -0.02430389470093108, 0.2779324669004907, -0.07677572628180071, 0.0774591607185247, 0.0780601733309031, -0.0888987611819064, -0.21668463861507561, -0.14717913934271498, -0.029328954310188157, -0.017986313123392467, 0.034584342396808194, 0.01369444024003909, -0.26602691492842107, -0.11897505120542361, -0.20844308055778465, -0.1288033508051604, -0.5192858514250913, -0.28953356792815016, 0.005405084088933769, 0.05872421821467875, -0.2720372853641893, -0.40383589974362905, -0.23696241137529642, -0.526274717119316, -0.16643762827186773, -0.39463502326985317, -0.48012912116200746, -0.3157729474501062, -0.2733939129172002, -0.5387507843786183, -0.4860751471300039, -0.6584081425070953, -0.8141969400478112, -1.16905819045306, -1.3715390528299396, -1.559174374673655, -1.496438205777711, -1.1938599244486605, -1.729129920467339],
[-0.12533891774942474, -0.26880413800743763, -0.3354121394356638, 0.04850256920652639, 0.07645507676446384, 0.15887203097360553, 0.14415986644805476, -0.07478856887467636, 0.1385816559797705, 0.23325717115302805, -0.020647141299575116, 0.14102470990275012, 0.12767509483324982, 0.021319967971088608, 0.2878742545317825, 0.1809102289151338, 0.18535905385088944, 0.24507554663004694, -0.08108314428533223, -0.03647271130631221, 0.006710100320536149, -0.22303878789810427, -0.4047937909380497, 0.18880129128148695, -0.1442293893383338, -0.42036967957144117, -0.41455007935950794, -0.26992391068742455, -0.39247481492095987, 0.0879339228366156, 0.6253811436570313, -0.5652177703278207, -0.6912154690117148, -0.6015853275636963, -0.5206741895875345, 0.3037394841343939, -0.5757846757963406, 0.27594319146384144, -1.0270897737614597, -1.9266955412926532, -1.354129521450521, -1.5478530929047505],
[0.4616444126098885, 0.2368376123327891, 0.050270765630080205, -0.22008169007522865, -0.28217083219705247, -0.31520304844252345, -0.37906462725056994, 0.14292454792110032, -0.05083994359396721, -0.073367132806019, -0.2104792239533831, -0.08846526225501326, -1.5858243956241578, -0.7237430302706283, -0.0315624765613811, -0.3338205415425489, -0.2605891217146163, 0.8477177686931916, -0.05762584381212834, 1.3171909061232354, -0.3633866768514088, -0.3543710478660312, -0.5382244412782594, -0.9015828074056931, -1.7982050661180606, -0.31631645726602964, -0.5564330012165967, -1.2576361666112665, -0.7853385281965645, -0.7791695987251062, -1.0946182353261613, -0.32051626276591755, -1.5236743765804592, -0.7808768490895467, -1.090478147310806, -2.931179452192442, -2.1368391831184725, -3.1191006865541833, -0.3528771567806516, -0.18113630638894027, -0.8885970874665509, 0.4578537765835941],
[-0.07224988419407896, -0.011771366931454233, -0.1270080266504735, -0.24388658299990917, 0.25353516019055905, -0.00679516808933736, 0.28290002711091455, -0.12603500590569122, -0.10257339281116173, -0.17614816518279378, -0.19649965398243932, 0.14974747839914163, -0.36287504110803026, 0.08237383254974227, -0.01960168248295849, 0.026138932259691446, -0.1854959667418699, -0.47624297922096337, -0.03529219534742621, -0.23239267199947153, -0.24048760011253978, -0.42330642378849354, -0.43236317844158556, -0.28479532646671957, -0.38229696746025177, -0.15845779864273551, -0.47258863757312347, -0.16889643942673377, -0.41128048564819797, -0.5119591825330649, -0.30450848469452596, -0.47669109627019235, -0.23389167185759085, -0.6725708221947609, -0.4114350129302848, -0.8190757824390719, -1.0211511292115427, -0.7221163369618638, -1.819689876984608, -0.48026070678572125, -0.8585804628069976, -1.2945281451624193],
[0.12613025142845577, -0.09587300465760565, 0.1020097324036671, -0.34750525540658245, -0.021747748786723566, 0.23043112543752092, 0.08752700768094172, -0.014948639852217588, -0.23249235620245373, -0.011338973321914345, -0.14785480948989532, -0.07466044914694901, 0.15391335117312416, -0.3033097598626641, -0.210903924614346, -0.1846757604063133, 0.11630311337310405, -0.525845410923046, -0.04172910165171981, -0.19166765222521145, -0.3296337308501333, -0.34014117258061083, -0.26733435878967166, -0.3986582706527617, -0.6733530002159176, -0.30647252359282634, -0.47229425736546526, -0.4866211857279565, -0.4908717845342252, -0.3980407932588851, -0.4185166042231524, -0.09715892184578077, -0.5365844234035214, -0.7983471270632798, -0.48792664017051784, -1.4660254146820728, -1.0394938780841936, -1.3751807332414363, -2.22601795321649, -1.3246616977245025, -0.9545429040600606, -1.494815247826616],
[-1.0884441432376533, 0.416119366946193, -0.0745939733840404, 0.7952682093947335, 0.44432432825649154, 0.606934040559021, -0.2959876084206475, -0.3099885311276243, -0.18390803388743932, -0.37547028287398315, 0.24765631968140098, -0.10870119608254356, 0.032006621704782766, -0.12856202927622234, -0.07282370797248354, -0.14261808177552873, -0.03877651037134231, 0.11635078836008006, -0.32081582438362544, -0.059303362265243825, -0.11619195251112192, -0.6897607916614, -0.7620588297338052, -0.41117634615402693, -0.14112506621555926, -0.3778136266753936, -0.20243077330519538, -0.12639009128193174, -1.0332636120957535, -1.3290663121674395, -0.4307072006455847, -1.1153266026864694, -0.5806161792639993, -0.7199875301597725, -0.8474762062489428, 1.299611463005575, -2.1292206746904734, -0.021956007767914506, -0.07289031712632911, -3.101299767253654, -2.330093515846261, -1.7203819190780725],
[-0.2447643216486926, -0.21292430903329634, -0.007612522858014558, 0.047840072695094854, 0.02428089231703144, 0.14559638284210447, -0.18971316186650783, 0.049507594479454915, -0.20118174795386895, -0.17308279612506886, -0.04249902229757975, -0.06222406692516222, 0.11961512804180968, -0.0633842641277993, -0.32074728122214163, 0.012911985838882591, -0.1152062082439968, 0.20115805488066305, -0.21276349066495417, -0.036776409108483274, -0.09236760969087675, -0.4102548120713483, -0.4326008015571972, -0.4877926069150842, -0.0651596057101149, -0.3839495203687447, -0.39513868776891, -0.41312143980464017, -0.4866277161391674, -0.2726657610673376, -0.5086843767149888, -0.6616763373207516, -0.4901982776258008, -0.5219447800945906, -0.6376145727462814, -2.0466651457788267, -1.7169809873552975, -2.1277716464249106, -2.1040026405058607, -2.359038459906109, -1.9490304005207277, -1.987773883329928],
[0.21479482440033135, -0.755117466308469, 0.1454667692867217, -0.23891467025340768, 0.3253783883034401, 0.15168183832389412, 0.3194098450176, -0.18378297032149163, -0.303595492052017, 0.030098417296486695, -0.6233388303646766, 0.2954017510307208, -0.04953995250129445, 0.12183021560574284, -0.08194332122927929, -0.24019991695356405, 0.2653490499976205, -0.43228384630321726, 0.2846332411386825, -0.1456765415721638, 0.1487328120817676, -0.32909612299757535, -0.957525951516939, -0.1462095550133198, 0.1275163742693942, -0.1687091139774546, -0.2956994810114776, -0.10344934911827271, -0.6764774765693893, -0.07551714890162146, -0.10726541923541406, -0.48625702016975875, -0.6431584114016038, -0.671942601831511, -0.5425614051608967, -1.862702840734406, -1.4070509687292612, -1.5641607111466733, -1.8211132980704687, -1.6489890973313572, -1.8814851656795781, -2.159393100266105],
[0.320641786483942, 0.11049805793303283, 0.3927101635660928, -0.10367450091182302, -0.41493945816827044, -0.3958495562713948, 1.0719749285915325, -0.33213778066776195, -0.07358362402608294, 0.27611492358447337, -0.3335222794202557, -0.5358826293891591, -0.4191490080135323, 0.6923902590593946, -0.10595485042299532, -0.0608884095056448, 0.3934290298395543, -0.3501629697893428, -0.5234815645267089, 0.15842708998861302, 0.030169283125050444, -0.6857145796054291, -0.4384574072942729, 0.04680066048932295, -0.5949866588412936, -0.5365805963188975, -0.703570420480263, -0.5171247124761041, -0.8559474674420978, -0.43632664877051447, -0.4574170778041698, -0.5623416832441698, -0.2779903248156727, -0.39705352746791533, -0.4017658683453386, -3.050712031150226, -2.420079377779362, -2.8763896451613196, -3.049150544193275, -2.5809995804946193, -2.6081002443139893, -2.7779014497765293],
[-0.6243436293237395, -0.2287491500700242, 0.3615616503495008, 0.1659693494222461, 0.12337204760252025, 0.15703214758290343, 0.25742105991319403, -0.2404934773667556, 0.08276022380555394, 0.04164901150625549, 0.0979809537608748, 0.01747207043093237, -0.01819806831105095, -0.10471717425397208, -0.15085223501507508, -0.11810147852625513, -0.06415615897938137, -0.11799245386532592, 0.090085385347669, -0.29713802456014865, 0.14582176628832538, -0.05935854594907545, -0.1897351803723506, -0.16916923299884168, -0.17203723286717154, -0.12436435236826979, -0.6508313873931701, -0.32901184598105004, -0.9105606739571697, -0.2884873834959753, -0.6541734759471346, -0.2409713852732024, -0.6289760314916705, -1.0439640769195189, -0.867822977903978, -3.4409928869269213, -3.111191396267277, -3.3566785990671257, -3.6422379306823536, -3.36634589077889, -3.309869465661817, -3.499940439675072],
[0.018458524287201274, -0.04344627951791615, 0.08390386044219797, 0.1685729484050059, 0.10239530197647626, -0.10442908184303212, -0.04162517963178325, -0.2045122078604551, -0.0949538706673419, -0.0111959608822387, -0.1380470492614279, -0.042376270350946425, -0.39482471946089115, -0.3630220785454867, -0.29149078767392683, -0.5664992750282221, 0.21111356676541618, 0.06313874679006007, -0.11572861073018935, -0.09476872524766385, 0.13791528408339437, -0.623077832853755, -0.5251128404971592, -0.4505636762371649, -0.24747553220673685, -0.02144265759490189, -0.6188954902775582, -0.30911861716822736, -0.7583983614064368, -0.3759776830047086, -0.841288077146936, -0.3348070252258144, -0.4099153144365556, -0.2922531397717929, -0.7483189143699696, -2.0823995770171835, -1.7935737993702172, -1.9792539323937788, -2.209269254595211, -2.0433334031822787, -1.7541134355950805, -1.9607318366525077],
[0.6352572814052331, 1.6406810440538042, 0.010168091259674472, -0.17058054725080887, -1.467896445871588, -0.9048590442385832, -1.0017583417939755, -0.7689229132389628, 0.5821219406974634, -0.6580200636783523, 0.15480041884076962, -0.9483088577020489, -0.5351766307844629, -2.950985156594982, 0.07305369850864131, 1.1778534734209345, -1.0941981093557747, 1.636667991025546, -0.9125538312529061, 0.14558204645280076, 0.21412171747526654, -0.3441481022705497, 0.7012019690748995, -1.6598116172201938, 0.9691994797159557, -1.1317946746755825, 1.01041089423457, 0.5477030346114048, -0.6584102827453067, 1.480855430149754, -2.4787492173494092, 1.0472845432144835, -1.909214096885267, 0.20317294182490397, -0.9604681479258496, -1.334855449591754, -1.544000240796588, -5.718735534407366, -2.194039195822961, -1.467380795249928, -1.522215117904382, -0.13872452605658223],
[0.2522107379268718, -0.06830639034263418, 0.045879445743897795, 0.21663571347009303, 0.2599179150966239, 0.038840647512016345, 0.18012537982619098, 0.145676508319605, -0.11372275814085055, 0.0639280140850798, -0.17519554196819087, -0.06653367883670445, -0.29518918826561236, -0.09574718368623693, 0.051024757974864, 0.26441968560165985, 0.22986922541924393, 0.14545709175690724, -0.06490795883943289, -0.29247238802293696, 0.14663328005182463, -0.646679052728803, -0.3726935271275484, 0.04536469505952974, -0.4943951306356961, -0.1921290416628476, -0.591478251999904, -0.2505266294985481, -0.8302356836005412, -0.2153952563347125, -0.5832494772416793, -0.5381502744725246, -0.7819167953913079, -0.23654143893678176, -0.6973710357037798, -1.9724383873644173, -1.381242218905914, -1.614203799588933, -1.8273804900049557, -1.6557477944711634, -1.5955706919817016, -1.8722014512720637],
[0.33066956211718807, -0.33089605721550436, 0.10290484658804382, -0.41421150009241653, -0.06228924327416209, 0.05422759469255984, 0.006799668299350474, 0.037842341076598895, -0.2700227238791227, -0.13435035055084305, -0.18414778617879207, -0.013759733532837213, -0.2772310412410814, -0.5509917946063893, 0.17184390504686634, -0.5928581926630113, -0.28572326191025454, -0.4899110241761782, 0.48210595427315084, -0.5144802649252049, -0.24798168820343572, -0.3443062398112854, -0.21820053499412087, -0.40817614195638485, -0.8085040730925084, -0.09692876735903658, -0.3134967909361945, -0.25067600764523973, -0.21887088381414094, -0.9439483847002095, -0.6248358838110674, 0.1055647400627854, 0.11308529603156621, -1.1020625715788381, -0.2554245477271358, -1.3962966939985206, -0.9360996945481761, -1.1730610514924174, -1.5906637118719713, -0.7569826048233984, -1.3288265849893974, -0.3104479206342004],
[-0.07135723883313377, 0.02200467036805457, -0.1374534852679252, -0.17612133643955513, -0.18781331210510455, -0.013033791675091989, 0.12719336352099517, -0.20004225418049207, 6.846811609631868e-05, -0.09937584460743572, -0.22599339413934913, -0.017106268494370157, 0.04147907653465735, 0.08712164509825919, -0.08136997499813345, -0.4674993353276755, -0.2823494820182356, -0.5220884856110832, -0.08743137958801614, -0.6224934880158605, -0.1349671778028465, -0.04025702970870685, 0.005597088255908671, -0.16840894047565574, -0.4671423486921417, -0.12894761150333509, 0.07259954046361693, 0.0716713414869722, 0.1430264926286361, -0.5903835880118523, -0.20256699476749224, -0.04421855269357378, -0.519423716987974, -0.6111616727057837, -0.25704653346341444, -0.4171066365422621, 0.019471642503272497, -0.8024708602459301, -1.2901479804262825, -1.2559793167359672, -0.021015058952185587, -1.27269396344057],
[0.3405319043344316, -0.7669015772472194, 1.648788359550614, 1.0411053435986097, 0.7388696747899318, -0.2452538319911731, 0.7049891368793203, -0.3072296782153791, -0.26878763161706415, 1.5237981529237234, 0.14891772372645964, 0.4705376943552426, 0.1612449834690085, 0.5603749970630734, 0.16458733537546238, -0.30109702243401404, 0.8319466243992396, 0.648197681973462, -0.3101718520443787, -0.7266761269075686, 0.1367018737690604, -1.013389816296129, 0.2110123574147983, 1.2917694438732734, -0.07792222011944709, -0.8735805867694012, -0.3768664903240266, 0.5334694094130792, -0.8973155972903091, 0.9067376166852683, 0.6156780581638053, 0.3366765859648918, -1.7340408735606612, -0.157295965445297, -0.08293569757761583, -2.472960037763719, -1.2561190881545965, 0.6287251481491376, -1.6993980543265401, -1.8751665695619504, -0.15902939243501543, -1.5739872634278258],
[-0.12218120464485165, 0.2361532835741048, -0.20623457218700794, 0.250270623084235, 0.23665872557458406, 0.09740440358776682, -0.012371562934583353, -0.07268600299516795, 0.07073793738445175, -0.23624861603829403, -0.14278091393368408, 0.20230716132456483, -0.07034999922902474, 0.04182255538809786, -0.0013939586315324078, -0.14666969351024758, -0.35663627305484064, 0.30664304081905797, -0.06442263535376054, 0.04718275592130596, -0.14329258289954025, 0.12927118400026943, -0.18111667473182605, -0.26856155001008203, -0.1315826631971016, -0.1886062237733094, -0.38336673421006046, -0.24493063036073554, 0.2941968430016876, -0.7004275176453089, 0.3888183528577844, -0.4292309344768473, -0.4794421710943464, -0.36208109290776036, -0.6940307797305332, 0.6198161129393092, -0.937130782939485, -1.0391204880030869, 0.35649308127548224, -1.7074273577113577, -1.2412363343586688, -1.516731659550771],
[-0.0759438855420979, -0.07593876296338661, -0.26321655213981415, -0.07807511853719563, 0.04221743991103158, 0.09078001067783886, 0.06340410813164096, 0.09101986440291006, 0.02158160160235375, -0.28804218435576395, -0.05010211839094175, -0.07525554283914347, -0.1261027176841924, -0.1996660014033618, -0.2795275204064271, -0.16912585896287588, -0.32601685057317753, -0.18618933160349688, -0.35370723722676484, -0.4881513944203865, -0.2530393000320914, 0.0893999420015175, -0.30435552857573644, -0.35564213442125114, -0.28318289075661457, -0.7353088719848592, -0.2292379975213791, -0.10621126747879288, -0.10010337915956678, -0.7078972780055409, -0.12132495420635628, -0.3655058748786788, -0.6689767085124815, -0.32103077868351854, -0.6885093889064952, -0.1752087616915139, -0.9511863199214893, -0.8581307849982843, -1.1961321418640378, -1.217663011852451, 0.5529884717478923, -1.5245834349419236],
[0.39953454392882604, -0.6718401831385049, -0.09331473053970754, -0.2961272518098951, 0.30086917233533106, 0.4646296999765129, -0.1233757414900408, 0.23254402888732217, -0.07450874144416134, -0.18550029629028045, -0.5069247915056982, 0.08207259526934493, 0.3746430435092982, 0.21140065460997104, -0.2600719698508107, -0.06400159160056773, -0.31088268851703615, -0.6029041390662966, -0.0689811318005073, -0.1710460667750562, 0.26237861112911764, -0.07827561251185691, -0.3623806147138076, 0.2521055520685848, -0.17545888313284516, -0.14686145648331053, 0.026657303401714558, -0.06439355098282341, 0.031994273550611914, -0.33721892984806, -0.13505955222806953, -0.22920400080836037, -0.7995491490051838, -0.7829472196506051, -0.509177864125768, -1.0833152390423897, -0.9168832188623837, -0.9868345494885912, -1.792246157274975, -1.7060361385204097, -0.3064388946819069, -1.4529128210375852],
[1.1361090696611005, 0.4999129095611814, 0.763799691262776, -0.4604654817769939, -1.607703543459081, -0.08327052521945767, 1.342719928234904, 0.42632040192011933, 0.3648329889877824, 0.311887342477309, -0.4430955869099499, -1.121327460038396, -0.05513368401971524, 0.6746714188738907, 0.4540014260057805, -0.21808664123042182, 0.21131371293898008, -0.06457181436661176, -1.2797088964096943, -0.4819790764895687, -0.02525838939163807, -0.4797131999850357, -0.398102216647602, -0.05880279945005928, 0.25095928837995585, -0.279099004644244, -0.5437628790369538, -0.5239598261630657, -0.9292886356548045, -1.117313271129089, -0.4389599811320046, 0.6435424144346488, 0.9745698951990962, -1.1647208388396406, -0.9276898716141606, -2.574808362629561, -1.2771886700938504, -1.3328433000269635, -2.7011902644712062, 0.44081255741896774, -0.791282636053984, -1.6329268524710079],
[0.22314988564931337, -0.0038180597574730647, 0.050539690224586875, 0.11159801335542374, 0.09145207330852388, -0.011135050322231483, -0.19223987844798368, 0.14000208921060353, -0.22562231448574352, 0.08611648931875754, 0.04655573391236142, 0.057443654243343883, -0.017671968751075684, -0.1764809056378199, -0.15089161164282314, -0.3363599762703038, -0.2266357568493735, 0.19876812196519086, -0.2874461353168135, 0.11234155696975737, -0.3294987401807617, -0.3062911521394017, -0.44792320096467264, -0.16602865300456132, -0.05523492777838085, -0.41939925183466775, -0.17533986376739544, -0.24451678120042805, -0.7561080923444045, -0.34160752512664055, -0.39087379918022674, -0.4792989718655651, -0.5306390440232137, -0.2332599734114222, -0.8314261852021294, -3.0600430729055375, -2.6935734139769107, -3.066069195409875, -3.174480925664597, -3.260417130532943, -2.976802629472826, -2.9396925439521464],
[0.21287145418187198, 0.46158544392476175, -0.5984749458695492, -2.3720267132067905, 2.2167758405676574, -0.7490977576952773, 0.6265161562759529, -0.44313850117933135, -0.19433462114555358, -0.9080911014066179, -5.124589911764686, 0.8035272287133514, -0.48442673303857403, -0.5495978321905093, 0.7395386339475725, -1.5701315566068965, -0.743032036192091, -3.068443039113287, 1.485307504716114, -2.503809536778067, 0.31483979198251805, 0.23415185092890262, -0.7816443857601705, -1.2227161359292786, -3.3116125537697014, 0.7345111567228703, 0.44449260237375765, 0.15211932097517966, -0.7366774020537975, -0.8872839470162477, -2.6140065934840044, -0.23072416953221797, 3.952264126342359, -0.7082731972389705, -0.48191858528905274, -1.0941840233589502, -1.4146132971901042, -2.3997392930078227, 1.9091022464580831, -0.7507122142708802, -0.9731479235430425, -2.059390891849653],
[1.6595492743092377, 1.7687839153764664, 1.9312023063785273, 2.6945216632745588, 1.2720309271401231, -1.6075085108168419, -3.659226529784739, 0.19002214978494797, 0.8350639846985667, 0.46727495229024896, -0.36786548505409994, 1.5200954344258024, -0.3534505672347843, -1.9267788402403423, 0.2819420734645298, 0.16436291273075304, 1.036915682777389, -0.24470122500975117, -0.049815961910180714, -0.45042445054286095, -1.0601817572101158, -0.32448276153332145, -0.3344719976067407, -0.5391844531004714, -0.932217465364953, -0.19167938159608333, -0.18998459427335646, 0.5104153789244931, -1.137877553256963, -0.8524890756619581, -0.8093410593706499, -1.4559784775527458, -0.9451623100832042, 1.6504873521300205, 0.37882841262424605, -2.7396483419389446, -1.87256380709541, -3.4509110647943597, -3.41807178486576, -2.6588167291581795, -1.6894656206106637, -2.658703848545686],
[0.08708468788778484, 0.1698759594516943, 0.20499430713654374, -0.28952670225634525, -0.06662800380933899, -0.08378989839484309, -0.0926773274236706, -0.0019353589757912235, -0.08862988159413267, 0.011098379833097024, -0.23361432962693088, -0.17652578647297357, -0.1553409903615857, -0.36708949162050813, -0.14666931794668742, -0.05791321648439225, 0.038076097756763544, -0.0984677951806865, 0.014359969556651659, -0.20123379700877753, -0.11704568423982804, -0.3123152794072049, -0.34571770540110136, -0.2228190491491161, -0.42213090736699477, -0.3462667893641174, -0.17494813981810575, -0.16782099524148633, -0.7954116710885156, -0.5858635292504197, -0.47447497451080173, -0.27929389959264284, -0.5521771258469764, -0.5252056078721598, -0.6151717080338499, -2.4155910934495823, -2.0662412910595127, -2.412612717205855, -2.825611838901997, -2.2718908865092615, -2.08857167359972, -2.22224228858668],
[-0.19935709405912022, 0.07640336324861878, -0.3236452815627433, -0.10572380072599373, 0.23388432838640139, 0.15055530985660945, -0.09867511258492745, -0.005487228412927086, -0.28226007429025285, -0.38368347608910625, -0.37921335081469776, 0.14902902673005927, 0.10429786729418787, -0.2841165060858658, 0.05772457677351259, -0.03910613892768877, -0.40765062801357105, -0.48060987797856347, -0.08817022948651226, -0.21639107754238515, -0.15567787800257488, -0.16854647305455467, -0.15340312118451657, -0.42390996445426093, -0.23389491642649562, -0.09884378411274154, -0.13367683303981653, -0.2693173714792824, 0.1155202763394695, -0.48239755630364334, -0.14090475430502142, -0.012299367469573485, -0.2632522298295355, -0.16670917278624747, -0.7325751988237149, 0.3523089255461514, -0.7008816301192201, -0.3940568830454826, -1.2281427356773413, -0.9230925050565518, -0.4922766159701776, -1.051561165014934],
[0.6463052130445367, -0.29239774539173224, 0.8890405588394996, 0.2184561786657121, -0.5317139857942201, -0.0945151831068052, 0.5235574601913567, 0.3601640186445946, -0.24799662051499632, 0.4708729306414244, 0.07562245722296307, -0.2933183053058008, -0.09602222035116209, 0.44335762906378057, -0.034765089301484316, -0.05189466929316249, -0.08612632798278387, 0.16857692252236203, -0.1034408210770582, -0.14408067028297306, 0.07581763466493995, -0.42506315381006543, -0.8642804038388995, -0.07570372980483711, -0.32219681300311415, -0.34374895178445003, -0.31411322599104213, -0.3879953861226885, -0.638815371195337, -0.36242845935326534, -0.24109143781279543, -0.49958263331419395, -0.17072376256207497, -0.3936021317136997, -0.404246551083727, -2.481180644474106, -1.7359618856436532, -1.7193061775600713, -2.1787046073139087, -1.8041772449855826, -1.7074232926794126, -2.036971930925304],
[0.029839322672712554, -0.10099492600506317, -0.3342447297471556, 0.03612846944386994, -0.14262528992700427, -0.03178159284805331, 0.28668314730401495, -0.07625682338793974, -0.1555035507167592, -0.2972164913525592, 0.033530282046925874, 0.08648373391537605, -0.16878621040270383, 0.22556824762443733, -0.15156449154340532, -0.1817654829524055, -0.003016751922627093, 0.05177188704217639, -0.25054452527398796, -0.12898760395357128, -0.08696050726589091, -0.0699821950567491, -0.1445012605557269, -0.08291901824123117, -0.2567620144252466, -0.367779962722908, -0.4377687206656611, -0.22545425395449206, -0.18443064233776335, -0.3851784824594961, -0.07524713388751733, -0.47181067540096405, -0.07522932362470788, -0.46318866984365226, -0.6994331060040881, -0.7134467712260022, -0.7731660616209336, -0.646204362416282, -0.7604526871220839, -0.9248266062617786, -1.1533827684391391, -1.2998953277890752],
[0.332872033889907, -0.6196149053665385, 0.9177531829785249, 23.718460171641333, 0.31762947768972905, -0.22819224482997585, -0.5973391348535214, 0.6361317798610076, 0.7735060337243218, -0.16206751270704725, 0.6920852594992245, -0.3813893811990799, -0.3962326643332181, 0.04233871947836156, -1.490070685254371, 0.11952539119554521, 0.30746800999478013, -0.0032477405784029916, 0.20729137098701966, 0.1253137263914654, -0.5169760906306148, 1.592910509338216, 0.11261909795201047, -0.942341169970906, -0.8324000718686462, -0.19939092701276576, 0.5691552979113039, 1.70111203483515, 0.8105069511075966, 1.2218926794165883, 1.6592783136552502, -3.3550048621522754, 0.7241883610523473, 1.5087552838703873, -0.5474932619872707, -0.2302967623853459, -0.9287237128993763, 0.46904751828858604, -8.685337430336062, 0.5246774835102425, 0.5667856835156515, -0.020865438842426084],
[-0.07394149522024834, -0.046062766988554535, -0.027450986484314054, -0.18259313335998292, -0.10944664676123375, 0.05472181135673667, 0.11091052154218638, -0.06077757793036414, -0.27856413635440747, -0.28757071496741216, -0.2833154108909698, -0.25178139162940355, 0.06299878876513157, -0.05544992976118462, -0.3435006942094993, -0.3590148702101198, -0.5325076033711111, -0.30560681816733587, -0.4932765408472001, -0.2890044396233514, -0.26727038773396294, -0.41578612615193483, -0.2587566774371773, -0.36505477708449696, -0.35915262302061746, -0.43149575290633496, -0.32311083139199953, -0.055701339435426545, -0.33981733862545077, -0.7451044876245216, -0.3088257873154909, -0.06951863574511052, -0.21073449795779625, -0.7690303072148607, -0.48163149937009175, -1.4497203510947614, -0.8424545160894846, -0.816638652083874, -1.3892262811633824, -0.9993368527442239, -1.0638887495874823, -1.0751190490341924],
[0.43577551196179753, -2.3979938479765113, -0.9182895239263479, 1.241997843588757, 0.3845394816627377, 0.7881661576417223, 3.0891376693251735, 2.055805003973369, -0.5881241734089441, -0.6286991523903596, 0.7406212258754068, 0.2949919076360488, -0.12167989521966728, 1.8087472084753318, 0.7319440075086359, -1.0627515914071424, 0.1920332885610735, 0.3620223915441486, 0.03945328051794157, -0.5004621113431017, 0.7714267569675032, 1.0305910183658558, -2.0257628376803187, -0.7932704504481747, -0.5531727388567159, -0.10353788955714889, -0.4196084350800839, 0.1372456536060886, 0.6689781481603944, 0.9764365897628513, -0.595464934343988, -0.27736562729174125, -0.7921137832115503, -1.2150956215230546, -0.42024828944651327, -1.1045550142981315, -0.16482950420682752, -1.0776269105336018, -1.8238816767181647, -2.1091112651622526, -3.285877315217258, -2.2676772479705716],
[0.026650488641018225, -0.025478528235284948, -0.342320046983583, -0.6107133455250106, 0.10257856368498819, 0.16857890631465036, 0.16914876796939465, 0.0466822983617506, 0.17689074850168693, -0.2126223934287503, -0.5553306048504922, 0.3002464090608719, -0.08499100650761918, -0.0029309423567224252, 0.1561655086238773, -0.08642099771054675, -0.21209380824929586, -0.6532129678317693, 0.21643390787926614, -0.8481283679338835, -0.14439388573555004, 0.0052674196824612765, -0.040125036596474144, -0.37392277158637005, -0.3820905822113783, -0.08640887355412301, -0.08393725648080201, -0.10200219232239835, -0.4391201359308898, -0.7946526236952064, -0.6486551412300581, -0.07107834250191652, -0.6518948712044715, -0.9840592680288044, -0.7891679708529767, -2.655732051888492, -2.2767071239235595, -2.487320781138092, -2.68337325879744, -2.7277591404216004, -2.2028871487883404, -2.850742225128453],
[-0.1054045167172385, 0.10933480737496072, -0.01595144784495896, 0.4261777856371196, 0.22610421094108232, -0.046069841447684744, -0.003762374459760052, 0.01938585083992251, 0.09929051301226997, -0.049264408635050494, 0.38662682577926283, -0.15117388897172254, -0.04210887090141559, -0.20719983304666037, -0.007227317572358165, 0.19174436787943566, -0.059565672543209144, 0.6232161813836359, -0.17319247618998376, -0.00048420224943773217, 0.07887231230369986, 0.016704619672928474, 0.21063238867504616, -0.24010492402130093, 0.2964988137463378, -0.15329148365752815, -0.3849045096666028, 0.003336734724765399, -0.8645872213232523, -0.34469644197218613, -0.15571180442163465, -0.05704088721397191, -0.47938238435946456, 0.1628273920234276, -0.20674779309207797, 0.6262787672520098, -0.31656739059638794, -0.1907289778465434, 0.1394463317119815, -1.1513201688520345, -0.4219920739831253, -0.7469716640615658],
[0.06377186316996204, -0.09489701888144576, 0.022948987691701065, -0.017191851604704, -0.15765478584767842, 0.20707089744776486, 0.006239927330358752, 0.0862668695266647, 0.11422533167381316, -0.26481181844623924, -0.05333798165078011, 0.03672464381805567, 0.054000946080003144, -0.10798535076450005, -0.2267599508074413, -0.1421219221835194, -0.10177062130072011, -0.10662190150806898, -0.07723136335526606, -0.11482886155474735, -0.12512824902165423, -0.188634302140006, -0.2540513407567399, -0.220924890212038, -0.22952081771025112, -0.37322444484791717, -0.3310300310906019, -0.13533536711597066, -0.3470670123691399, -0.25037268818455893, -0.3306438956213629, -0.2772582763961904, -0.2851707761282552, -0.44686543317048294, -0.4434908017006129, -0.7291135272987493, -0.9310103570243272, -0.90724503638108, -1.415270260465349, -1.2716676204233148, -1.1042561402560804, -1.1035553671593106],
[-0.05240610077027, 0.006776290642132735, -0.022928734496429556, 0.04952222435056358, 0.12193763400545325, 0.023663946724326382, -0.02054757636725107, -0.1377068494181351, -0.3761677128444523, 0.029142099165261334, 0.046467260979716415, -0.17114748142942277, -0.41177128877220165, 0.0626346808615993, -0.30632400466419146, -0.23099414654202136, -0.30991768153169696, -0.12589377003211696, -0.40685385774262717, -0.05381907312842167, -0.17376739426678145, -0.2672273453555462, -0.3192096500078638, -0.21712537307515217, -0.5617033030509153, -0.39086765633800474, -0.5741893765263525, -0.43330463499754296, -0.7110587198928958, -0.3837141297156523, -0.5652533280687628, -0.457912443033053, -0.496804201270893, -0.47888299365917014, -0.45061065771243203, -1.9333413934543713, -1.5404549309494404, -1.8679807120999108, -1.935143212905222, -1.894531495729842, -1.7682692180107542, -1.8812191876412783],
[-0.131630339881153, 0.0636735461124208, 0.17417972853654376, -0.1904345799626809, 0.1746547516084901, -0.05548695804479098, 0.18771201229859494, 0.015585814517572757, -0.08061738437383785, -0.2668544055080495, -0.14713965524611008, 0.23189604120791504, -0.1000320908335661, 0.030781391155746907, -0.0882665330777396, 0.03716243545811135, -0.2015115946639188, -0.4538664052037854, -0.4214004258675569, -0.32455150921769865, -0.09803105337183625, -0.3271927589451549, -0.006433958789007647, -0.12715459373648144, -0.372545902164082, -0.004960766955588529, -0.6078064411083989, -0.4483212736493941, -0.3866103161673054, -0.054697700947372925, -0.3017706423667926, -0.22315063260872073, -0.5804526574034579, -0.4382526065056544, -0.5678299401265803, -0.47328188428404905, -0.683274975615377, -0.8583205667660392, -1.258112388693908, -1.074063361061584, -0.6816746468449175, -1.415657796398806],
[0.4023097397172695, 0.15169025960005547, -0.17984283686864322, 0.7498555035374717, 0.14339329620380784, 0.21169522638085525, -0.23010879332856168, 0.30773260077320713, -0.35814624837025066, -0.6707706669472163, 0.4482737805829364, -0.24404233974301842, -0.07682150990150276, -0.40303051517583177, -0.48793084057363156, -0.2016743833074, -0.5814664547743197, 0.38491381892224014, -0.39796221547691946, -0.08838618182837088, -0.07312195384929167, -0.46365337707073057, -0.657075755048477, -0.6422394806588662, -0.2809058321170481, -0.3941843938039469, -0.30229665299208786, -0.5632334304349004, -0.1840910041713123, -0.45000504513076195, -0.2908554851584209, -0.7773683487476614, -0.6813902058279285, -0.021416255678897645, -0.7280475983425926, -2.2221182434018285, -1.8167593967795548, -2.1836922180732024, -0.8967059273105381, -2.499078010672022, -2.202063858239386, -2.0838048003973366],
[0.014589726390503469, -0.04132180866079765, -0.07108866588675174, -0.3024046365348737, 0.03598046386358937, -0.048040530718439965, -0.03115157418580136, -0.2113609921239614, 0.0482631270445689, -0.2861893022968044, -0.19465407097486348, 0.02797310547853252, -0.1939747221059624, 0.07535447443011707, -0.24931534692509627, -0.09376281415181885, 0.0464437605973486, -0.23896183398622217, -0.007971329691046436, -0.39657629844282494, -0.07583748861498348, 0.15586855793890786, -0.17024052432104209, -0.12708257326774755, -0.4584202152464105, -0.07081817747989601, -0.1212183167085484, -0.21819384120772758, -0.012591526582832394, -0.24298658553723823, 0.1865500717845957, 0.06100660036162167, -0.3512416746255542, -0.4226562853761144, -0.2437712188728309, -0.23550601860126272, -0.19639918083241478, -0.015395425111203532, -1.0001343972858994, -0.39598964727445946, -0.15923303037763917, -0.9588442497471384],
[0.2653570185110705, -0.2810525118571884, -0.18275880645814385, -0.0987138000290128, -0.27741921697545163, 0.08739911629004249, 0.12471259168373908, 0.05132880626050797, 0.2776679570780964, -0.3437026682645962, -0.47085334833457004, -0.006555200300853788, -0.4560367871971341, -0.07759418129748724, -0.1560226977147686, 0.3833288533923859, -1.5509138172168657, -2.0026235173453593, -1.5853116902382165, -0.4167257401510099, -0.561398196667099, 0.08871734307726804, 0.6541102372185352, 1.6648635051230198, -0.1490843443630337, -2.087185368732456, -0.8688753995464197, 0.15081633578375772, -0.18966170852527378, -0.6276150706491694, -0.7617694343814968, 2.386487620869468, -0.04294797123442689, -1.7820217571124268, -0.35363911940401044, -2.483100478489495, -1.686053149711563, -0.9534782941892265, -0.3611790585321281, -1.0013728322343352, 0.6753182820410872, -0.4899637208527014],
[-0.011737980203250908, 0.1263801656324091, 0.12897294946951515, -0.36222436809340586, 0.031999505346582975, -0.006722137231947588, -0.025234013553117444, 0.05945935909703917, -0.03851431788933224, -0.14810060700334793, -0.28433389280016863, 0.011148213531751052, -0.12887727691223097, 0.005947938019135766, -0.0914699380289105, -0.1841441195053356, 0.15092009589707342, -0.09813867827976823, -0.17543636345984043, -0.1960142721423405, -0.1773992133430871, -0.247340359345965, -0.24995126861235167, -0.3583457116386807, -0.3141771994338423, -0.35121256079669483, 0.050070629188299604, -0.22697423127353836, -0.8853078724711285, -0.6008302111925946, -0.5445210193188731, -0.2552767180799751, -0.38535199081113314, -0.4606818868492627, -0.7243697206319676, -2.6687282806292156, -2.3415797474858104, -2.6330846229416185, -3.0160132253922636, -2.700112873924695, -2.3925944135088915, -2.567961519232188],
[0.017422996320901944, -0.20228984525707444, -0.05646071990946819, -0.09633274000223215, -0.016694989416274963, 0.012521003943304231, -0.030338105292120127, -0.21662659457290398, -0.2938003514877998, -0.12749764708335776, -0.21614183812430238, -0.13180956828385382, 0.013031225500546945, 0.17562266792235748, -0.03536390399051604, -0.11407772628622344, 0.05670031464804352, -0.17561199683625658, 0.06462728857844875, -0.2772267228324157, -0.20804073729159783, -0.4405228436265862, -0.4269964689180384, -0.14391141460069384, -0.35894271440659636, -0.17213066079371628, -0.13546355382361996, -0.18171557010583686, -0.6648045230227686, -0.286219545336264, -0.4322275131152795, -0.39033036739586713, -0.45433790954211245, -0.6792907802700768, -0.8140327340688321, -1.836489892493591, -1.5980306773941078, -1.5514569853713045, -2.453470843293111, -2.00998889921481, -1.8453723254750451, -1.9406370404111464],
[0.029984638367632172, -0.03408490510113269, -0.12549698271330378, -0.3016955609957721, 0.02640927761036435, 0.010429239977310423, 0.13448818451049852, 0.2155304141495626, -0.29270887685957603, -0.34838440413979127, -0.2558596891022908, 0.3741130562684643, -0.0032040060918922817, -0.08676902765210788, -0.06904037657995572, -0.2706331131380826, -0.31005939256346027, -0.42601799238297433, -0.09538276505596498, -0.2969757693505143, -0.21599484889061663, -0.30337454141446213, -0.25346244000489404, -0.4881198564743499, -0.38567844286446246, -0.4037509677671759, -0.3447097778592457, -0.26029658897818037, -0.24086554857653256, -0.7901025595654412, 0.04040567304562923, -0.013057294125332687, -0.7014434859790738, -1.1529543283872048, -0.6964684657081871, -1.116584541175218, -0.9248805966464612, -0.885099032398658, -0.847680534124155, -1.0077408820279263, 0.5220342329576458, -2.0144374890866352],
[0.0740627384476959, -0.11494369957773345, -0.0973756703264467, 0.03290238393682734, 0.06310017642499426, 0.07538417501829472, 0.17963760310438803, -0.06587455373244172, -0.04007451618033869, -0.368452424455056, -0.3105058056080494, 0.11891873896435874, 0.06960330469737937, -0.09884733114506505, 0.08574316657560083, -0.4641812110861907, -0.3518014534015093, -0.1440769224684202, -0.19830053777283957, -0.2173075970944007, -0.15778477772967797, -0.00328423551395814, 0.08588522180804206, -0.1277303888257894, -0.4770952191903706, -0.3058619548656902, -0.30217550648885966, -0.1318059063323566, 0.37296024489013807, -0.062140524394229874, -0.38962120503095343, -0.3576834017721189, -0.22369412453281876, -0.5290695406927465, -0.5206114758114987, -0.5131059140385255, -0.25481266359962496, -1.5607221770663326, -1.3532793421075466, -1.570873583886981, -0.2764694155412281, -1.1272928873046009],
[-0.13751557260339495, -0.15554455336450665, -0.16968378725150782, -0.22574588797259112, 0.0785534239700845, 0.041061579983445, 0.024414182656632408, 0.09531361968821055, -0.35374289250815866, -0.4645669849893961, -0.5472264947030664, -0.007376995372728795, -0.14740814314920433, 0.05276679021221798, -0.22220007601583733, -0.650043666399781, -0.2452305212952527, -0.6781152265184279, -0.41371679963918967, -0.11019390087239396, -0.061047793724531084, -0.25185712634904184, -0.5549646618742102, -0.47624381996166754, -0.6540842543911395, 0.1486931362336801, -0.31699205843183886, -0.1471581277800727, -0.36721558178702796, -0.6478055377714285, -0.34211374505861397, -0.1568619722812089, 0.0025455962576312412, -0.44351436897396457, -0.33267524063103776, -0.5628144179205226, -0.30960792967616785, 0.14402818509911325, -1.1576445667655941, 0.015080860415062113, 0.0416532301298324, -0.8237341455309494],
[0.5855783429664121, -0.5043056040816816, 1.8214392566034086, 2.9407863839765898, 1.761327192114278, -3.253551081700134, 2.6410859897929253, 0.16771639280089504, -0.16401608320729374, 0.3797239635602182, -0.33549958210913233, -0.05937301609341589, -0.24335171957431972, -0.9287990605742749, 0.20447804175149642, -1.0071782370881834, -0.7257555413844444, -0.7606953143907331, 0.10197188964300792, -0.5863944698611575, -0.48299238469060923, -0.8185704535334588, -0.09897905292744401, -1.1026255447721736, -1.5612158215807685, 0.35861132104641336, -0.48528987575423066, -0.4114112428460436, -0.4785109089489899, 0.7338751117070691, -0.5466893340440999, -0.7341332682598551, -0.31824570065634444, 0.2332671120187724, -1.0755487399213732, -3.5561195780237163, -3.7944439774378242, -3.4055283134319216, -3.4324870687819287, -3.709534948069203, -2.808425085359653, -3.3641916591569263],
[-0.10500656581440186, 0.25568611045290135, 0.06686944220616778, 0.2717256677485801, -0.07790201238241017, -0.0052536823944670415, 0.1812433025916714, 0.030518536692271517, -0.00934305951373225, -0.15709820451948348, 0.011762615897822468, 0.03674859746680416, -0.2196639615890043, -0.03120255423299094, -0.15339831676835913, -0.10640000810184011, -0.2928411037785574, 0.10468839482681068, -0.22181286887093551, -0.2256595907235887, -0.07076642666520931, 0.0003180923576777297, -0.1526421297418499, -0.3011256585370617, -0.24012539121755463, -0.29227471653007303, -0.41044423982844463, -0.25204031283244144, -0.5438697480244727, -0.4350565611770716, -0.2574372911548643, -0.6076429743196973, -0.3437682078652119, -0.20074004346994523, -0.46921900137306705, -1.2932949357071575, -1.1832049959499527, -1.5124743536887912, -1.1911877227496293, -1.401284506989173, -1.1522712835080473, -1.6174169292739426],
[1.529198531398301, 0.40582264857680445, 3.605665356733346, -3.0552060956186295, -3.0761106239961635, -6.68399593930478, 8.411887685832554, -1.7398035127589409, 1.643140117472297, 1.4703548297115505, -1.0036717059462588, -1.365685242820113, -2.224689738373854, 3.0439334229495434, 1.647134549260529, -0.3281621170545374, 1.275469251572856, -1.7917955797589156, -0.5126482041831208, -3.048329836197423, 1.1615213474747015, -1.555390362066369, 0.7783940885746441, -0.6034249716048101, -2.8389441972928022, 1.4544291232398305, 2.7793984816990314, -0.709632596194744, -1.780294887899688, 0.9439581102566873, 0.3198335946732584, -2.198592031974473, 1.0797521902727283, 5.459027533301773, -3.0908089282690034, 1.4135379166146753, -2.578077653399447, -3.063340930849101, -0.554684581975893, 0.4820098744141777, -1.45313456616884, -2.7187668606271966],
[-0.264517035192644, 0.11379634779962515, 0.16821080743049088, 0.018887437035463087, -0.03305577640991127, -0.08044736230827837, -0.11718695342537999, -0.22228531097139337, -0.10706643042476714, 0.07184615267036638, 0.055488364518590134, -0.06310840195013515, 0.007111210336423927, -0.11274333772007249, -0.16121628075541386, -0.26353363511262046, -0.22029877207952073, -0.23826006665503047, -0.0629909163328318, -0.0899243918862659, 0.10286270727408722, -0.20406765530629967, 0.41570898609047313, -0.09300470018161838, -0.12695981875510745, -0.157858781591681, -0.25341848810438944, -0.03454245190995443, -0.7847725756631545, 0.3664716764588714, -0.5105436328330127, -0.6939454426842415, -0.2583448884812346, -0.24841580089756374, -0.1779079265584078, -0.7668120836347466, 0.11075210422420595, -1.070705566421258, -1.5637967811833688, -0.19522778573516095, -0.8672416757907045, -0.4588359783960938],
[0.445634376869198, -0.2722501579414244, -0.11389161462804868, -0.48583592942291504, -0.4977078308183644, -0.042318198287819186, 0.18724426455544663, 0.12081919414741364, -0.03867692755167608, -0.1492216846143374, -0.4853323852911005, -0.3981150390541051, -0.01222164223876941, 0.250876786918009, -0.3918954199865825, -0.23281480548503491, -0.5184601526842529, -0.49488061263003363, -0.9118307328264849, -0.4185492157488518, -0.8564225341461053, -0.23725544863294923, -0.57458826669032, 0.006869351730802673, -0.4096551651702497, -1.5299819295722845, -0.5904826508723083, 0.029572544479517217, -0.2828790888501326, -1.3356346486980069, -0.64704890928922, 0.2701285773225815, 0.17279219726145648, -1.3168148911878805, -0.5310731447111275, -2.3398789210036144, -0.9665636045915716, -1.4882172973319254, -1.8778716903962553, -1.1534959344552358, -0.9988567838652415, -1.1378913229602021],
[0.00924580206549492, 0.11342038950911325, -0.37251693998485835, 0.060447045355624264, -0.1660991834559394, 0.15717734859802787, 0.41690144871961216, -0.09269106049874491, 0.01596561593326644, -0.06865940280719088, -0.054240817451476055, -0.26668745029634505, -0.08002648386796749, 0.2399331024434432, -0.3561548046289615, 0.4611909773009611, -0.5416300539868092, -0.33972061600057096, -0.35981290730498916, 0.04980382740434095, -0.07184211960716477, -0.3440135760126125, -0.22099557705074924, -0.6550547054913372, -0.9114369792844443, -0.5354013191424043, -0.3798598002919481, -0.2896802971443894, -0.23225044453795185, 0.2388127080130108, 0.012572669747158292, -1.037990281636816, 0.05342849738453981, -0.7759033970866189, -0.7810460056959764, -1.9155275806581418, -1.4577223461112838, -2.3144175907595246, -1.4372380721676745, -2.2672141187874755, -1.855038454483971, -1.7264516066206508],
[0.029754400338719084, -0.1700354334259331, 0.34543274786976474, -0.20722676902629303, 0.059328100754769166, -0.14703331161587727, -0.14188042583934427, -0.14946766489554386, -0.0029519763905401764, 0.07682078371877861, -0.021447811678065247, -0.016662997893707336, -0.11651132331285279, -0.263188994972587, -0.16020889005409744, 0.02006164106508267, 0.04894622685228166, -0.3108806825581109, -0.14302865498443768, -0.20503110565696092, -0.14459316954569781, -0.12449987203072999, -0.416162647284534, -0.04132406534371852, -0.2189603741870681, -0.3703300892951006, -0.21935342162436663, -0.19468164867180882, -0.6842926412645487, -0.20509054114580227, -0.6560969278983401, -0.17484426306790699, -0.36358064612480995, -0.2553134124441744, -0.4655918338744343, -1.678028561196318, -1.3324129792663184, -0.8133441720702496, -2.465946778236043, -1.242153953935332, -1.1929506597669486, -1.046375653520704],
[-0.19342885337407448, -0.010068772364141524, 0.2947499436992018, 0.03701923373558123, -0.08453560529645372, -0.26712816415603574, 0.2540102548434861, -0.04110813259690234, -0.20546387861820992, 0.11620065727030744, -0.0317846645299984, -0.2814273487742857, -0.14224647304430627, 0.262560599477252, -0.100355703587657, -0.08199518771388105, 0.022147987299501085, -0.08828133312961434, -0.3821886872258041, -0.1085231538520983, 0.037529089769709735, -0.22929416192357663, -0.20572750195260228, -0.06038345083716478, -0.4514039679173564, -0.1895178818701631, -0.35340077337908216, -0.3138170904773038, -0.8054054105129423, -0.43351185110881657, -0.38170497323537295, -0.4441114562587969, -0.5463879881625845, -0.3987553976942729, -0.7461604619055513, -2.757558169883071, -2.4700340912283103, -2.958199355393034, -2.955086636998597, -2.9241859926467635, -2.718423290712943, -2.865100127771823],
[0.23009971827791337, 0.02032898439903954, 0.09779644401913261, -0.3494905471386057, -0.02361192195088238, 0.10842993175035091, 0.038704098155948156, -0.023413951737676825, -0.05580089427556753, -0.11290255495586991, -0.2651810181035564, -0.17808948909834846, 0.12633074823014037, -0.16791119674095129, -0.3449282520796506, -0.5888850442068811, -0.4267186247998319, -0.5674522145529892, -0.3957515759730428, -0.26152179058314107, -0.2510040765806066, -0.059007823042710204, -0.22702419696936266, -0.6935991871810182, -0.6384998805466467, -0.5508194746399417, -0.2937645120473569, -0.19076701293961865, -0.2738961499425683, -1.1135412161227298, -0.5784425526799039, 0.17894567416053536, -0.4174563181740497, -0.9702017078681187, -0.8070489264759526, -2.384710434058148, -1.1987679246755123, -1.7125432760038053, -1.9607489814442298, -1.615427073873279, -1.4868686193824265, -1.1606540579889015],
[0.056413972814120236, 0.3648461081178569, -0.5030945824570049, 1.1305497914118676, 1.2431469255996421, -0.4518465986255042, 0.24947065701113882, 0.37134501159479655, -0.1850659408811781, -0.5661124892750553, 0.5674800552128031, 0.7936217779921079, -1.255382237456545, 0.18249114326337842, -0.22317348553107533, 0.7244148661689419, -1.4910304275557362, 0.5204251772117114, 0.04927019232958273, 0.19671623757262072, 0.2999634056043527, -1.1303580776243047, -0.5177446876459374, -1.099585761110972, -1.1826179263194434, -0.05221529633297196, -0.4026220713743554, -0.7307016959663913, -0.6304455605528657, -0.10029718000604504, -1.5056304182807707, -0.5748333027518865, -1.1288999239073798, 0.7812030370813169, -0.19323430094873073, -1.8484489887299684, 0.04644387373938826, -2.3864910191847977, -0.11086675943472131, -3.090205158643712, -1.5789971533901381, -2.1336110980314436],
[-0.10285413217229943, -0.037203645552551776, -0.3835959711194885, -0.6642915651881202, 0.005701488825261555, 0.06335245995018521, 0.29651181010847233, -0.041061357545905036, 0.1004855630962036, -0.2967468928832292, -0.6317043013889413, -0.12373508224836072, -0.3101381997729651, 0.0064115757836438695, -0.11494008914109177, -0.10026342097781676, -0.08955016150776461, -0.4732925754342072, 0.05767707535573147, -0.5223617273851242, -0.3191650633370046, -0.2587914153532938, -0.3253798696138365, -0.20255812572836693, -0.4353363540729443, -0.2867036120550224, -0.5205741914610844, -0.3892228610891356, -0.773969963259031, -0.5736080398767489, -0.6626923908145252, -0.07866725675324743, -0.48336883424943056, -0.7851852253683519, -0.6248158433296203, -2.6004616236343994, -2.1276565728601216, -2.517808870675722, -2.8084404125816187, -2.5756000912907178, -2.2209672925427943, -2.6856664965623263],
[0.23990307756337992, 2.4264193716138083, 1.548938455769572, -0.8252869014729465, 0.9199179583782464, -0.9226116996927624, -0.9904003631835576, -0.21213562527278224, 0.9496580288201824, 0.5173940301919511, 0.05763484216967139, 0.723406286516224, -0.11046040109208793, -1.025769260782331, 0.5249424915033003, 0.4012027055724673, -0.2921591637885191, -0.3700892339628476, 0.3160659298673908, -0.587514433740958, -0.35414336432545185, 0.38972820653719153, -0.998669617483567, -0.8668376537600017, -0.6820761144384996, -0.11467137980955165, -0.15191783713971121, 0.2946883693760408, -0.7560530197161962, -1.3135496707134464, -0.8003795718920267, 0.7047660345136348, -0.9507398437337042, -0.2542195544061558, -0.7433740243623957, -2.723180817519794, -2.7793192947803513, -2.9956460288776285, -2.8927402649149414, -2.676102310072399, -2.5650956153115794, -2.927533355057525],
[0.1606588170541809, 0.17027961490903845, 0.04069845505246395, 0.17822394955293996, -0.07602018461019511, 0.07424394624332223, 0.03283636286544816, -0.09661702983203585, -0.10532426275403244, 0.03697366191100308, -0.08017146542204774, -0.09084319571197322, -0.08683842065324496, -0.008828050784963763, -0.06611395397150686, -0.06761755614977462, -0.13503706101909244, -0.048148954162059644, -0.20374973395019888, -0.250581436050768, 0.05959019149551829, -0.03512021159632194, 0.061130490429576974, -0.014507530767128475, -0.0744119024400115, -0.13577458678488202, -0.06733392321160786, -0.23057768303706747, -0.3534788322690986, 0.3782956126422951, -0.050632454594130044, -0.2894923715343934, -0.20535022857121862, 0.10840162576090037, -0.31919976680419837, 0.29654483304941404, 0.016207600357314052, -0.010160487076254257, -0.5257515913443473, -0.5803810013978241, -0.2214454161805347, -0.7340665945511048],
[-0.21280533274579516, -0.03808357566079444, -0.04973636448355387, -0.18396008949264542, 0.1008081629544835, -0.08846250037161452, -0.1070218117839449, -0.05253997370668869, 0.032250490378151975, -0.33779352606832236, -0.19529743630553054, 0.10972798224234924, 0.1095381216453023, -0.19456193103938235, -0.0741329375783775, -0.08450282110056195, -0.053903105306673735, -0.5171152672889511, -0.26702367524000975, -0.3028581161624801, -0.005352224990392568, 0.2178646359675815, 3.447834273974382e-05, -0.010474049073619018, -0.143399346148771, -0.22781711292645862, -0.05663324497325006, -0.13912706602228173, 0.25309766130013606, -0.247050493750234, -0.05359964679655633, 0.30040529291160906, -0.3662946457017347, -0.1727967324124859, -0.3412270044456925, 0.20942429488158967, -0.024081043993666115, -0.3371537501916552, -1.208694536333158, -1.034761293462853, 0.23457606297787414, -0.857207982735988],
[0.2165862433788767, 0.21282276204124276, 0.2601787052528286, 0.2747886647056306, -0.1422652520864052, 0.10562362005420171, -0.005676937949057545, 0.11090565032040135, -0.26427265251784476, 0.1160703785858753, 0.2521963951698063, -0.020980250488452836, -0.46973462692614465, 0.031858961308007805, -0.08146961325030308, -0.26438847923986664, -0.5514598675073916, -0.04979270557710285, -0.16825321638405338, 0.3314014995413804, 0.428936156660667, -0.4989421796619036, -1.0298099059069228, -0.5802225490456091, -0.602370451442146, 0.45935006570609316, -0.6340451142051062, 0.23102536528095108, -0.943606532876809, -0.24298385926513877, -0.43777851651514327, -0.6091186395931815, 0.7721178385721266, 0.9610723084677323, -0.08571582676780168, -1.360290721887323, -0.8811439475039028, -1.0676003718704294, -0.8506750287806253, 0.7906585495259387, -0.36980991701997973, -0.18540267833308763],
[0.1313560045896924, 0.07027235148401437, -0.0486836997008534, 0.05834670116778549, -0.005798612881036744, -0.08650717661664145, 0.19635237528128766, -0.04070301914507656, -0.3940433451275061, -0.3455961892095985, -0.16535710865523662, -0.07643086427550272, -0.11948699866329178, 0.1860437796163409, -0.12624051478533257, -0.3751454431585009, -0.5454754319175715, -0.3026119449255755, -0.4115169775412207, -0.17024639717456042, -0.045043427426856984, -0.07038420162208152, -0.550116428611353, -0.36452985283412676, -0.13874731107548968, -0.19999309284845523, -0.376866505428583, -0.36868861789013274, 0.08210219642702989, -0.7569304429230683, -0.20286463802437227, -0.2859846947191115, -0.4953612999281176, -0.8064276806974887, -0.7467626651792321, -1.4037298760876484, -0.8546951524113411, -1.2760503260262748, -1.1677691422196703, -1.526582939005969, -0.8976844320439563, -1.8118200850518051],
[-0.03264980857263796, -0.24411047901938154, -0.13925824328824055, -0.0697114294185283, -0.097940523160866, -0.2441390007997831, 0.0025457030390448735, -0.17511516807250435, -0.17496023659010793, -0.5704282974675737, -0.18359080798111674, -0.10542950179046734, -0.6874489056351681, 0.12853506652414026, -0.1801863190647031, -0.501836670525927, -0.773302796733918, 0.12528436303629734, -0.1266468424574046, 0.18146275157479178, -0.1545742319560368, -0.7181986607254512, -0.680648983470436, -0.9204215555770859, -0.5620958265925204, -0.09775583369563011, -0.9416605383209659, -0.7623984515147416, -0.5082689193091905, -0.582159341486649, -0.5744770305582573, -0.49502777284064126, -0.5946864593114309, 0.022388663019635865, -0.9954648458745603, -2.359600846343659, -1.5877222650402676, -1.9430095023352894, -1.2550715838065007, -2.3188877270789017, -1.0282604010805423, -2.050979043792995],
[-0.039236575308769894, 0.18957723886820763, 0.02759642694754166, -0.029667011682341593, -0.07009109494024408, -0.008104894758684594, -0.2314380080652614, 0.0004305226073059547, -0.1947300348584876, 0.013488138342547423, -0.20854160145485967, -0.2416045906378977, -0.16241620903528922, -0.12695944915958743, -0.3348319794020563, -0.26772219222765176, -0.3436861807991098, -0.23844850779200447, -0.26713841948959366, -0.09667924107644799, 0.1977266127075584, -0.20796541137957575, -0.2969794417236418, -0.2793361003485919, -0.3654290319638869, -0.45524224713490985, -0.10475672929597803, 0.01748932204530934, -0.6154949383658003, -0.6282207012052969, -0.5084971083744516, -0.23961190930694456, -0.6564579996786511, -0.10539313546177076, -0.573806054922359, -1.2701274660771253, -1.1936517534075228, -1.2894219863044303, -1.2779187812098707, -1.0632431370392033, -0.9318539122391496, -1.0053581988136218],
[0.14251617187669507, -0.07595403432266673, 0.1916499183943398, -0.10773367510262369, -0.1838183212257028, -0.030644698764994902, 0.07726306302393215, -0.17109037281027237, -0.17930875401607482, -0.2629448657582862, -0.23253138523828873, 0.1875101516640564, -0.2613033836952369, 0.26843836699991575, -0.1097196605810684, -0.26528112325342734, 0.3561768438840355, -0.48559158714997935, -0.17408467385136417, 0.24695179756020375, -0.2714361084807123, -0.4806560012109516, -0.6159016782432486, -0.130004142219109, -0.8226113174324337, -0.49652833783192923, -0.2953441916012368, -0.27255760877015944, -0.8953712940467506, -0.8077872166825762, -0.841866245928607, 0.3582644656801612, -0.4485948101521556, -0.5363765589524025, -0.6947823313434863, -2.4327869080258564, -1.866569631081814, -1.7800100938498213, -2.6205133201266784, -1.6314966265236184, -1.7981859187154614, -2.1280159602004196],
[-0.028155699642950456, -0.046222843160966066, -0.15928617970398382, -0.11974745328224179, -0.2413932226823347, 0.04795203173145688, 0.007899587297719713, -0.1787125151206303, -0.19279418741187948, -0.3205990787459229, -0.23372132087246966, -0.06832157596045366, 0.0773278512431438, 0.002473494252261412, -0.14338974492710727, -0.38628661973454964, -0.4375806160726275, -0.46869393304643503, -0.1305604121784238, 0.2984692969288411, -0.2610073541168542, -0.13964174212449032, -0.6140596004034689, -0.4009853682622051, -0.8224523135867975, -0.28239644859144275, -0.31667304680892117, -0.5365765563304544, -0.4036434277577818, -0.6230802444247222, -0.5279860896840651, -0.23775685902602703, -0.47482211620448805, -0.5506410713030405, -0.6405990247006164, -2.6115375560743264, -1.9515178933129886, -1.969839747779217, -2.517202934993054, -2.0704957274341536, -1.9482380897049012, -2.206377161744381],
[-0.0008220568820830499, 0.06411583731505585, -0.10597578289951358, -0.16022478733311069, -0.0605102864705644, -0.1371768984544295, -0.12056017345618658, -0.15429480709536753, 0.028460673608149478, -0.12557896237671332, -0.026304790188269107, -0.07195837707953534, -0.04720578658590123, -0.11759575114079045, -0.1712946322356544, 0.14547473334156671, -0.057977121382892655, 0.038946687636335395, -0.18975033901728958, -0.3810039900936936, -0.10046215727594195, 0.0036483215712362453, 0.0253120600467264, -0.13536329932213068, 0.038797184259670714, -0.08884951340664614, -0.170159895516495, -0.2790862499755871, 0.5353108198714952, 0.0035096917095910497, 0.5019288984456874, -0.0058025239074829095, -0.21869097102738025, -0.49293714107388764, -0.6261831526978695, 0.4929645210855477, -0.03749211430803726, 0.14296870201174175, -0.19405867655578468, -0.6614056820573654, -0.16581055717676657, -1.4017584031215793],
[-0.11043025808580229, -0.10000745310982931, -0.19423485710489183, -0.10967449011229574, -0.115217123463677, -0.051552942272136325, -0.04103023706610306, 0.005424594555932286, 0.030031772603221644, -0.10404291156252411, -0.26745093853379764, -0.16040928310254163, -0.08880354445963007, 0.001814326537902004, -0.15249303972414438, -0.09253137908429834, -0.18084726762602565, -0.38809255043924196, -0.25323638010541205, -0.29804141015156066, -0.1700374434173255, 0.03173430844887836, -0.13378109269802474, -0.14794336721712883, -0.3025791461889356, -0.2437997282375478, -0.17239166878621962, -0.1567076277033456, -0.2999955279770699, -0.24397388614198992, -0.5029471024928558, -0.2716112526150935, -0.4071764555407011, -0.42327342600526585, -0.48856218717618394, -0.6374010227199425, -0.6366866729530947, -0.8886509094793732, -1.5768924035230787, -0.9096663502853737, -0.5572020795156709, -0.9030673757129482],
[-0.01627448895116907, -0.02895934615321995, 0.01957733825906107, 0.07351424178820266, -0.19089815089913392, -0.19140359805719637, 0.000569781940872645, 0.10318822864907308, -0.03897224538274459, 0.10230166679512065, -0.21616231534034286, -0.14098944952504194, -0.1990881385395369, -0.0389272927725041, -0.20836029977978554, -0.22801209348072576, -0.21704703290869562, 0.03431408146985986, -0.1882384327649479, -0.12128627252585213, -0.24612896234237946, -0.3460225109096996, -0.32726010085116447, -0.44175226051975164, -0.4808530658633907, 7.357904496740275e-05, -0.3364207429422717, -0.20732590813193513, -0.4405481240911128, -0.3769845651011714, -0.4393851336368898, -0.518667522306666, -0.01702903907214164, -0.19987498658957983, -0.30052260263737246, -1.0520398576689851, -0.25718710637085473, -0.651227076237534, -0.8868657516280449, -0.30580821594737095, 0.035749817233983655, -0.6707207429732231],
[-1.9968355009386425, -3.533709692628826, 8.161611860985378, 0.6087044379853536, 2.8496611876166535, -5.442241012429457, 6.046707789040966, -0.07503746223233412, 1.0310305974593097, 3.1663198003083957, 0.23773429756645292, 1.1149107552005133, -0.957096016413552, 4.025951905206899, -1.2789809842000843, -0.7841617792420469, 0.36904107874264364, 1.0417428360786039, 0.098705874140309, 0.5097725649246062, 1.088818244996391, 3.46909217845317, -0.12707371864485975, -0.6440726665020159, -0.015304419178939798, -0.25036364596753974, 1.183822479080591, -1.6210921382656656, 1.2861908490536673, 2.469321559243234, -1.8827401688483936, -2.373561291892128, 0.6206798099636667, 1.5254562147085058, -2.8143416798098637, -1.6347279608641172, -1.0753463423599532, -3.922298620500358, -4.66273694239757, -0.678717667112646, -0.005982691550151079, -3.7791034659404725],
[0.22567927078159108, -0.1407774709766994, 0.28762856887818755, 0.031004989916789554, -0.022362900155990555, 0.21845895039587965, 0.05215776871810938, 0.04356806355314594, -0.01322745642815503, 0.0981765861895421, -0.09799381176351611, 0.12469249870875902, 0.06323956055369359, -0.003945429999256353, -0.021981411681077628, -0.04459273696561661, 0.28254001901848064, 0.05731660405317287, 0.027895648251577008, -0.23712145290891054, -0.16975355479044224, -0.2571647469446516, -0.49336463069797115, -0.22261897140980044, -0.2451643749745694, -0.292668527946383, -0.21552501560047635, -0.2490337656788613, -0.7721332751832517, -0.40581584295175516, -0.4245821692435614, -0.2918371649578043, -0.34560958852887286, -0.5792710972220917, -0.8995457295963625, -2.516321042733089, -2.1382959580161196, -2.2577407681520576, -2.6994338095486916, -2.5360123528114675, -2.2038848771427206, -2.3765673848676365],
[0.0379352774945801, 0.16254966137524868, -0.15890274442862912, 0.08173424873488468, 0.009201850999513188, -0.12511833071633185, 0.24681298796404413, 0.06357620394356979, 0.13569515881175848, -0.07753265813873969, -0.2094856120230813, 0.060425622711976254, -0.37115508946772724, 0.03550212373674317, 0.1438600092352435, -0.1592395600859209, -0.129707578678343, -0.3492357570226575, -0.16094781387830767, -0.16515331705496958, -0.155534031665564, -0.31500386647163586, -0.4394735141831861, -0.3766922831591542, -0.5830809836093239, -0.008261060593507848, -0.836109967990897, -0.26856094437713457, -0.7639351821214575, -0.20077102698340366, -0.6178874210829223, -0.5883491537605716, -0.28456270311026866, -0.4706188180879476, -0.17454949814081422, -1.5824084562684084, -0.893820829367298, -1.385998929248721, -1.7100080881123818, -1.3208877169599622, -0.9585321493799926, -1.232601026950783],
[-0.043865003928779255, 0.023689277218310906, 0.6845265701229465, 0.3033162474191728, -0.2704499277806669, -0.013621434184479097, 0.09242799023327285, -0.08217527567254475, 0.12534195017425145, 0.5564162288143815, 0.03170950336117217, -0.5399346027852959, -0.18241432820790202, 0.2601604486812928, 0.2247899001015157, 0.42641651084909643, -0.023824459360239872, -0.09342670466323674, -0.36007766020271437, -0.3680335859177969, -0.017443610734151734, -0.5589130014941496, 0.0016948701360856386, 0.17122619970055036, -0.3726498216667631, -0.4886167379998708, -0.7927665607397361, -0.17013292521613235, -1.1951461626580409, 0.662461057586693, -0.43776014678441416, -0.344082317153971, -0.7472905081008009, -0.1860541394175765, -0.49655602516283154, -1.3277999943615093, -0.49701934605609777, -1.0525689144268817, -1.6808462945651177, -1.0912344603880226, -1.2789722850174883, -1.0476465753769213],
[0.48976098181734273, 0.04625013583927609, -0.15479622450233435, 0.07850481314927861, 0.31423990851556916, -0.23915273884774269, 0.015393901111070668, 0.36067975263213115, -0.006270849619291838, -0.0997581576223526, -0.11196585762375588, 0.19678501898221246, -0.2616837440225373, 0.002525981692858913, 0.07398384471522926, -0.21325451055044445, -0.09002589577884086, -0.02830669169076331, -0.22225408595520899, -0.11970919969526225, -0.21201481232800695, -0.3865700611707051, -0.5022986372294379, -0.09977524808065184, -0.16310782936171062, -0.27319639842049664, -0.1871718329833737, -0.1837434565235184, -0.7115166535212509, -0.4692855389284634, -0.32893321119719604, -0.2509009080793897, -0.2528795998610407, 0.024998335250725194, -0.22356412964896613, -1.8011456458351718, -1.2622359765474722, -0.4446099152079358, -1.8971946904534507, -0.6674439722877101, -0.8510052857569732, -0.8565914646504503],
[5.61303752551176, 6.3689408387361555, 6.035224378507758, -1.535022187494333, -1.2968972975666129, -5.056766570322343, -3.814981524801555, 1.287905702334813, 5.418508974327776, 3.235428285390886, -0.3655574630476915, -0.2706601843362518, -1.6139500656092816, -1.9321761047394195, 0.5735019384712561, 2.2387671731119223, 2.7832942426108542, 3.388902898673908, -3.5163340669963916, -1.8329581930496701, -1.256395175898974, -1.2980214164423234, -1.7455598149319693, -1.6615361891858407, 0.9000339531156257, -0.9481472733510121, -1.3342818223584756, 2.609354292682355, -3.7488799124338468, -3.1691982793123255, -0.5299477308388577, 2.0799596152715805, 1.7969953504264045, 0.49298497452405654, 1.776277144347177, -2.6823820799260942, -2.3179983259013075, -4.463066607242357, -2.1352946798641295, -1.0584612863245102, -0.8152522615707674, -2.3337599965700218],
[0.3783752309273066, -0.26925905805261485, -0.07135438692135515, 0.13987123892054687, -0.16134612378228552, 0.1186694223547217, 0.5134359949199555, 0.1975405825037905, -0.47106713784020965, -0.09234601997666748, -0.43557482138959586, -0.10504174554788602, -0.01333859138461942, 0.328676001951972, 0.15060549730411946, -0.6854648523822456, 0.1750764172054134, -0.563442553566653, -0.23750482379434795, 0.042678143651646384, 0.02199358751163952, -0.28041240376537185, -1.0083771346817176, 0.36537186004791844, -0.8090282153645532, -0.4579987310627341, -0.18333521170465697, -0.2560886432419324, -0.4291730182356728, -0.5545742165530101, 0.0005331236847189833, -0.5584510638143384, -0.7675390398969373, -1.118928878268153, -0.5916997408125261, -2.325930573885732, -1.8587046732709653, -1.708495254387342, -2.6648034659968753, -2.2625304784741473, -1.9864665475809056, -2.230143644144486],
[-0.29612397725828593, -0.1861806852920056, -0.1197558453596678, -0.030883556460419445, 0.18953054910154887, -0.04652240096418896, 0.1988740874496356, -0.36069413366732966, -0.12455889368569872, -0.2560496288671991, -0.19681728365486545, -0.16470350731181488, 0.09743558031417933, 0.11436873886268631, -0.3273046911174764, -0.11489034871752324, -0.17856097169298013, -0.042612764694886306, -0.15848976265132766, -0.2803058037112985, 0.05088518101076629, -0.3557449518642716, -0.09754560957014481, -0.31528392743842687, -0.3896756494899538, -0.2804699572708243, -0.26823866745840824, -0.32368122141986627, -0.5330333165937774, -0.27134877171204197, -0.39944371783287924, -0.2728681598791119, -0.5648236353912158, -0.6495950149750528, -0.7923945736326903, -1.7056085824384628, -1.7802535405832751, -2.0923964037092944, -2.364030298400854, -2.281081730863959, -2.1001991657419987, -2.5110634385509805],
[-0.06069787207924032, -0.08472744498392093, -0.22410571940755566, -0.4821181987070924, -0.12918384539285604, -0.05685354964269523, 0.048127194827466496, -0.09823324780241541, 0.7268030801319374, -0.7736356968607624, -0.9991530571600751, -0.34208638727849955, 1.2212716582706902, -0.20232328367858324, -0.44785005959748875, -0.20523237975942435, 1.997196610310262, -3.0835828895146093, -0.06116798340157759, -0.27823388699055984, -0.6450222987083285, 0.6644589606098866, 0.9942635531496065, 0.06190831021219803, 0.6406007213768102, -0.6766907433560465, 1.0718858974731496, 0.4067236757707723, -0.11530235353254019, -2.0150211679045094, -0.2769156912748192, -0.9156340205210023, -1.5799181253884163, -1.1434392549573775, -0.9840100899749746, -3.2056524531034416, 1.5143443257798654, -1.7848349388215146, 1.3016393290421926, -0.2219160036652563, -1.7983839683528533, -2.658160829209266],
[-0.1539167750385967, -0.0042046436418503325, 0.10236521537657868, 0.3231174946525954, 0.12154745876111964, -0.09429962974461042, 0.13545776983773242, -0.25630541740737744, -0.14256788510617763, -0.158702008168659, -0.18367428470018976, 0.022924909371963675, -0.2892512009115382, 0.10416048273737817, -0.06313740412716498, -0.15282045255615553, 0.09845316429388609, -0.04693568721444418, 0.13979101830030036, -0.17219766457497415, -0.053374856117775864, -0.26909781371882474, -0.3240987175012985, 0.13618075722224854, -0.031164702972287715, 0.01484623094268373, -0.517798820017343, -0.39202944097810244, -0.9850448700119505, -0.4161030081004984, -0.23549591062953099, -0.12769436769239823, -0.4673161222117944, -0.37249884019853463, -0.8750383504163677, -3.9038223421662193, -3.5951956025579004, -3.928091786135147, -3.9872884421183814, -4.020467153476689, -3.810434100667416, -4.00318190168552],
[0.06729343222311081, 0.11513833915204191, -0.008450508840517375, -0.0978036254638027, -0.07858015062217942, -0.033208258412431406, 0.13177553070887627, -0.0554891174642299, -0.22046168872969002, -0.031146094735823127, -0.077460026551796, -0.2289261996020659, 0.04749400724023165, -0.12510727946033368, -0.15099371799399514, -0.12876205029513094, 0.06561031290114289, -0.13670821335339461, -0.19565963825465935, -0.13285415809803994, 0.05342575335089909, -0.40113161557276095, -0.19914743267975688, -0.09594615563703605, -0.30024433103049875, -0.17589069996805629, -0.31489818079461046, -0.27910562297333186, -0.74897993555597, -0.5710058622858747, -0.5717876662849191, -0.3680721931490382, -0.3778626199269761, -0.5100716274457117, -0.8121457802317182, -2.9285585521908537, -2.5616599668343834, -2.8121499843280136, -3.018257516885536, -2.941996518420903, -2.6579620411100393, -2.8539019834114128],
[0.00982184468976981, -0.04307887775284377, -0.035788637645491804, 0.0997397660627905, 0.08632796931627802, -0.1409641436163226, -0.1542090463484863, -0.1850168654514162, -0.23268875052977117, -0.09761153942555667, -0.019498033046974592, -0.19090715065261651, -0.04191383440499459, -0.06721827974507817, -0.05880681175332931, -0.3076461263548984, -0.08464704475985733, -0.32426126861654003, -0.18034505888122518, -0.2631701128019467, -0.015621392389623732, -0.2446778591354121, 0.13055063345358908, -0.12381454576056554, -0.3703416698965216, -0.37150901252917307, -0.3661411378177179, -0.14550978318948943, -0.8982739980792556, 0.39085540291269444, -0.5652946358508792, -0.45565305760029984, -0.2686303685721389, -0.19691318357162127, -0.4690599968332504, -0.5336256849793508, -0.4490954062040707, -1.3287660215502886, -1.6418347072954713, -1.141816702854721, -1.084996632746226, -1.2108480675223279],
[-0.16990562903300116, 0.3420488428637751, -0.19478348275601992, 0.01732876325093218, 0.02158818030984397, -0.21612948727739506, 0.023904979798716437, -0.006585943946309299, -0.08319942897288934, -0.11754228062182752, -0.16408519555485357, -0.24454844521354788, -0.2790908222411697, -0.06393046107890217, -0.11755607725877479, -0.05299231343952646, -0.19185622898694643, -0.18925598567330595, -0.2600681340147717, -0.00012317948212962697, 0.01706248158189726, -0.14415400645386386, -0.3593004836928248, -0.30180390779173555, -0.3246799503184774, -0.051054500466614354, -0.2967166223547328, -0.18031247563100816, -0.5704683420472954, -0.451725230763166, -0.1473101913863441, -0.297070444612192, -0.4488432955177526, -0.365293785234347, -0.6780199235203764, -1.454339427863356, -1.5639342556349705, -1.911150603552012, -1.5856649644370093, -1.8465713481651675, -1.7339008050391314, -1.9017646818964722],
[0.11799288219001006, -0.02143250011009339, 0.33882965094652145, 0.14875211614266656, 0.2626836029847292, -0.2375617531967097, -0.08453388265269918, -0.03395939499907211, 0.11599935150737622, 0.10907755872424105, -0.18098764574049175, -0.0669510961806019, -0.3250081628331085, -0.3394700087095587, -0.13032145838120968, -0.12798193991626156, -0.018400864023640544, -0.3678923267868688, -0.047708785719558326, -0.29963917860000616, -0.11429557257411431, -0.20855267685914483, -0.5558569265561736, -0.12012150647598703, -0.3693962082048156, 0.014244550396673042, -0.4921382305886859, 0.04538390237948121, -0.49550115831459196, -0.6317714672226306, -0.3693164575979431, -0.4556363258429322, -0.018871702300599837, -0.2451847408709061, 0.0655210232970796, -1.0212282079339403, -0.3700326671038479, -0.57101936935227, -1.6412798811532898, -0.02113208903848644, -0.47264639753385623, -0.13890775495681487],
[0.1639979110634622, -0.12452698153074303, -0.026578357677615974, -0.08396926947573773, 0.08441170052327325, 0.2680820312055214, 0.19966258890789876, -0.19496652139166573, -0.4077703008374287, -0.4745903880852054, -0.15737735290941265, -0.1431770396273357, 0.047911902070819475, 0.04906007559894011, -0.13699563553165067, 0.10597349610860861, -0.693195110169974, -0.04863417357789573, -0.14840030327306195, -0.2605357049845291, 0.1575785096456405, -0.19544560002899947, -0.825311219676292, -0.30411835954140815, -0.592371031312891, -0.43015516315454566, -0.52039819474235, -0.47993609284872274, -0.8835866055155718, -0.2837658694764519, -0.5571676503842027, -0.011304066300580975, -0.4937339105981466, -0.5769248867611969, -0.6235767276599493, -2.358861918102407, -2.010410545105273, -2.3921891216666142, -2.7770983077598252, -2.4160954717718526, -2.2994090593326018, -2.627945422412207],
[-0.03345211424485114, -0.122364237143227, -0.11437543679058254, -0.4453911697726687, -0.8544431200365333, 0.2844731464906452, 0.20257708886243814, -0.3658529974386898, -0.08890207107719704, -0.040085142692553144, -0.5543185119090317, -0.7170189388349175, 0.2462968312392155, -0.17176902979547126, -0.21683519293500764, 0.08219429440886457, -0.4378994781799418, -0.6692312234058726, -0.8554069207640725, -0.23017698442877688, -0.5342266589532788, -0.3847537244262463, -0.29773527706044367, 0.31360840347425517, -0.3538032349105264, -1.8149811405901148, -0.49965498937680086, -0.46109625861552184, -0.5722265115981016, -0.711330377253975, -0.7672307066476043, 1.0708102851979402, 0.8813171030191471, -1.1239087995847334, -1.1550273450853672, -2.0740106297974377, -1.5612234972921815, -0.8302552763799622, -2.0949036552873332, -1.135145015679117, -0.9214151521480289, -0.20937422172768194],
[0.047387945942363535, -0.33786970279978706, 0.28110220823147114, -0.19465241369413236, 0.06990741661489623, 0.3149255020258597, -0.026465671755911236, 0.1412705851269641, -0.08379272704783125, 0.13863854003178097, -0.48653308724853483, 0.11514231891178223, 0.08686818316215085, -0.25873171018968555, -0.10217679357383024, 0.003560207707716949, 0.1943599598624037, -0.15348301384383933, -0.1705094485959868, -0.3534752216553484, -0.253285593201649, -0.03916061093075455, -0.11075685714896355, -0.09169500599109427, -0.13000515407903565, -0.336681265380594, -0.22094599829122372, -0.36410288550765446, -0.8269712533215643, -0.3794508023427539, -0.4701423164009916, -0.25000680456074653, -0.44391975533259487, -0.5927771507769478, -0.8479814519472737, -3.2235914364953953, -2.867817931006699, -3.224516717224806, -3.4927167755422888, -3.394118184264419, -3.029814855761544, -3.2769229822699284],
[-0.1840985504547821, -0.02308613779551145, 0.06407901556799409, -0.220558990626051, 0.07996266500500203, 0.011969723773741624, 0.09853816951279838, -0.07744493517276256, 0.11526132960229275, 0.006152514522274071, -0.18554089653904782, 0.1297356480376889, 0.04004565529882413, -0.055573002416578074, 0.004501252516644127, -0.06669234525541308, 0.0534521217308479, -0.2990697566552207, -0.0658747580596389, -0.3155471551742702, -0.2305227386768981, -0.14307775484199398, -0.36469145498273015, -0.3753213198176552, -0.44426149614018995, -0.2209672703956179, -0.3242792085176794, -0.3514039304267578, -0.6510874600590015, -0.5104348188995939, -0.4886905226534172, -0.0996610826348628, -0.4495673542381914, -0.5610410166700002, -0.7821674080695432, -2.2427711631260987, -1.841668836374144, -1.6831598426450896, -2.515215090679988, -2.1444409764193506, -1.7398653163334086, -2.1937337630634244],
[-0.27968801667059, 0.19612213633713993, 0.1776431028410961, -0.11081284739990833, -0.16725543955736016, -0.3546856774469066, 0.1519271062112225, -0.20603168481681594, -0.14361971853095204, -0.12494781601498597, 0.053379872283694355, -0.08703978819670036, -0.3129392997991305, -0.026015459631333913, -0.029132430255833078, -0.18268525683833647, -0.17414801202862154, -0.21596593206955778, 0.011929679018982583, -0.2513716391514795, -0.07185773496440484, -0.2579184234650753, -0.1578379850052008, -0.2863278880270475, -0.32185372756436276, -0.1437457190010898, -0.526598741164655, -0.11124794907590696, -0.7515477367514867, -0.3678515209829566, -0.4757014718799221, -0.36393637074114094, -0.4823087871297608, -0.21430928303641375, -0.681117927797647, -1.4521442977476564, -1.5743206123557674, -1.8687741747894604, -1.921731696486286, -1.6312751334111848, -1.7152882813501906, -2.031906054177053],
[-0.1553778145011602, -0.029168963570283875, -0.20989401974788666, -0.17580432988019282, 0.0027153649949931947, -0.14001984064396342, -0.030825954036894542, -0.20496116613121948, 0.11683945505341327, -0.11040945518485647, -0.23031934386476163, -0.0606537765103087, -0.03950587900334649, -0.16664143652164057, -0.12236324650958678, -0.06759895803441313, -0.0564318672157533, -0.18942738284656518, -0.04580549734212948, -0.1633506654919999, -0.1842944353407218, -0.16075150606095828, -0.22846601998489122, -0.3111024339471583, -0.44367861090225974, -0.2161697942436293, -0.3823099816249046, -0.3369502375288805, -0.7369819360588994, -0.600787338679167, -0.29260881521941196, -0.37556935500396077, -0.4947805599345857, -0.3490595445279996, -0.6367333902458888, -2.0712119881546247, -1.7976401102186184, -2.177436901428867, -2.4048009555302445, -2.0101170380316185, -1.9119593928754526, -2.1343955052069066],
[-0.11824695850636521, -0.2669695337013634, -0.20953645853920608, -0.21163305202047686, 0.07433307026393793, -0.11145768203003828, -0.061252505231101456, -0.24541303212491983, -0.2146567692363606, -0.0039970218687855955, -0.23718622884032478, -0.014479622791459363, -0.13863958471275062, -0.016629306399698653, -0.27807816493909443, -0.29545141387178125, 0.20403574520385478, -0.6357644583771975, -0.0530370796398199, -0.4085202549195427, -0.24400201891299336, -0.2839423778368861, -0.45750275958570025, -0.05997812560080107, -0.24388306777407195, -0.17004568868710454, -0.28058266172971524, -0.30070218854695946, -0.6625082163992949, -0.19873848429315025, -0.28474673810731704, -0.41243426383902143, -0.4090387209514134, -0.5849942507919195, -0.5619692819893439, -1.3977793476743339, -1.3576841916800044, -0.9021275519834502, -2.3000499766640057, -1.46850768549648, -1.5598532191426966, -2.1467556786305146],
[-0.09782819802241137, -0.18636893944950383, -0.1727317140639487, 0.011769194953162367, -0.10198252307676141, -0.04648774513104721, 0.030727384238963693, -0.2871368280865965, -0.1375018212711921, -0.2881931852929202, -0.05431976196666219, -0.10860644625405917, -0.2373347183498279, -0.15227650234921195, -0.07198977505381697, -0.1551837820254847, -0.23197185557960426, -0.5388891529002217, -0.2530742570883448, -0.15107504084183196, -0.07653239809052963, -0.35912028881361635, -0.14193774416257368, -0.5772665425614439, -0.39951121316548577, -0.271259510135614, -0.4738396784667757, -0.056159667448253944, -0.6210112625224155, -0.3744837093376726, -0.4969581533066345, -0.3047726109689725, -0.24852329142705631, -0.4078021950649696, -0.7767557006409932, -1.9422525060873475, -1.702923057868728, -2.0071467783125403, -1.9345412331999103, -1.898443505259494, -1.633676045681382, -1.6952284151465882],
[-0.08178734178475229, 0.18825046325396358, 0.06387651159638895, 0.16661766353018181, 0.006030787596513659, -0.1966385424140326, -0.17039345025463748, -0.0404198082622109, -0.2159037369875289, -0.08777536069738223, -0.12704808606191426, -0.1981076925235787, -0.10467923684887076, -0.1547028586626386, -0.3688770612587648, 0.05301496032822059, -0.5519560450698853, -0.21674126433571844, -0.20283206960837308, -0.165506562232382, -0.08821802518624725, -0.03424108125763572, -0.3280510482733229, -0.44472925953016634, -0.23195574165027052, -0.12558120688444238, -0.8591525484084495, -0.24241752576376835, -0.3246930120796039, -0.7468161554105668, -0.5681955530367611, -0.5839286771393757, -0.03572954254695573, -0.20652733237263574, -0.7146424652953225, -1.9206488018838532, -1.341135502697229, -1.7038285237591992, -1.5699972751211217, -1.1226161028658825, -1.2770481970766638, -1.2871049570988211],
[0.6453544582889031, -0.43962982246148286, 0.6435346155785334, 0.5228693317984273, -0.5165364816201687, 0.13015620652146237, 0.3645725667928911, 0.20217368001345984, -0.3896372341777633, -0.11062816654778353, 0.0307022105398425, -0.4889734561368893, -0.38503948201621974, 0.5598196533449578, 0.06432506000763577, -0.4827472905511749, -0.1419769237520182, 0.10797820985213309, -0.3593596988616699, -0.3854517529273324, -0.11489725294829146, -0.3158725674666267, -0.37098556777355224, -0.15226134733420016, -0.22766657334752152, -0.7347621304610346, -0.714936910037842, 0.010015385217492612, -0.83137269245602, -0.5091958790923659, -0.810480394142149, 0.14787728204867576, -0.29916267977372346, -0.79158346405738, -0.7281301150617975, -3.704433440340733, -3.1095326906494525, -3.560720838938593, -3.7903476114312236, -3.444148870390647, -3.4016986489489813, -3.5973472945986846],
[0.03274310305223328, -0.061261428820748476, -0.27208061474534745, -0.11030894151973832, 0.08792638553812719, 0.023238671705338977, 0.0015692032813048574, 0.04448419134784019, -0.18592858521465183, -0.40594032227311605, -0.3093564013575132, 0.10912990461589357, -0.3740271280245674, -0.28684811438726965, -0.1319026778719648, -0.151196294308735, -0.5150626004304744, 0.9247758098511667, 0.3231009541098614, 0.028300798729322937, -0.2482884098060235, -0.34809703992644675, -1.1192407012384096, -0.9056418493921521, -0.12797670022188537, 0.08500687988606259, -1.0508893897914453, -0.6081276745707824, -0.8592241088303237, 0.0391522104391649, -0.8679390104764488, -0.7810281550722379, -0.31322079473487685, 0.21046114748966954, -0.5455995784388306, -0.24147174661412657, 0.19189840468204675, -1.3829463991143083, -0.037035358769476084, -1.6566656924602339, 0.7400943783040026, -1.2267009393938788],
[0.003155587848635416, 0.15324330200690747, -0.08107540295379007, 0.2503214959241925, -0.09187253015177832, 0.10767229215881623, 0.052095465173589726, 0.09064226470181999, 0.08292064022166189, -0.32322291787876145, -0.11768547368917956, 0.06062281927951771, -0.22029408288126617, 0.0631319388532409, 0.0011292827734840508, -0.16771165250102166, -0.4100773532424242, 0.041146780063729266, -0.2898729283817148, 0.05829799394813021, 0.03788852389985852, -0.08591050873097662, -0.3120427157059925, -0.43475733719950743, -0.17397005862500817, -0.11409181583327588, -0.38548208151664676, -0.4626605544653033, -0.683026401794756, -0.5120326996996308, -0.14576050343533212, -0.49508142021568763, -0.5382252202859072, -0.5001531304263215, -0.7787324432502837, -2.7664208642796493, -2.5023094986195726, -2.813724864797378, -2.384722392806542, -3.056664883971798, -2.694489142599226, -2.8848673315506526],
[-0.1922245489422293, -0.026852926643106425, -0.017637763611574635, 0.1914562196232123, -0.03091949814581632, 0.05314829019686425, 0.06475466409035284, -0.08114980023446815, -0.01255315990366548, -0.0789588451211839, 0.15569543079504233, -0.25143094051667325, -0.22227727821703055, 0.06103279889909174, -0.4511146379392687, 0.394615373927282, -0.32354016620968284, -0.10927063219524116, -0.4609158201855771, -0.3026828148595931, -0.1311666758450586, -0.4104921273337309, -0.45202900274771834, -0.8883033177958816, -0.5816658359492768, -0.46959402398983097, -0.750173179066612, -0.49268531443558905, -0.6552401713421391, 0.0787707597478928, -0.6841817512250303, -0.48477356174524616, -0.40030728168316165, -0.23904250299440152, -0.7781553906373172, -1.0099706930517893, -0.7454566735893585, -1.4039923521906152, -0.9391972494193943, -1.2694351432709319, -0.9040847087237279, -1.6761180882888285],
[0.1175599212475801, -0.03121164423744745, -0.1713494267804806, 0.11377501659234189, -0.35229731037316453, 0.292807701304696, -0.0753603162400802, -0.11458268490135033, -0.4400364675898396, -0.2574159933350901, -0.03900494918892141, -0.19158983999770005, 0.26301068028012414, -0.20649521213460797, -0.34298412778964943, -0.05916865904582804, -0.31576169666887827, -0.1722290110728509, -0.23993513912666303, -0.16081890192290024, -0.2507754945862924, -0.5452995131275707, -0.6235451003287378, -0.5324827330408742, -0.5614649738361275, -0.6528666664648638, -0.3651578048000707, -0.4233163449883528, -0.8676978418125804, -0.5643501580335559, -0.6935131270546363, -0.44377219674030316, -0.7692680255041773, -0.6134451729039194, -0.9430554324885778, -3.7667332845728083, -3.2310141234044294, -3.637646745381676, -3.770193071888545, -3.649947879430177, -3.3785775480968234, -3.6288708378682286],
[0.061846729829299604, -0.020243546850899366, -0.44224800742174475, 0.23470566770605497, -0.247744473326063, -0.02908208773788919, 0.36297315547203257, 0.00862284230344043, -0.11971880646755902, -0.35669466650179976, -0.10907528749634272, -0.2450581737321911, -0.12339617391540451, 0.2876808979940205, -0.16709785589522555, -0.033579487387843256, -0.2810542102684428, 0.1192205044593306, -0.27630410993519244, -0.07526729473033301, -0.2187071236960068, -0.2009996254302993, -0.3656190361946158, -0.2826048582299674, -0.110901099015404, -0.4311839379589255, -0.4371163758182144, -0.10218599141908513, -0.47976888932655504, -0.6611958853771992, 0.010137113517408532, -0.42161411912560237, -0.39497710726176555, -0.4728090898154796, -0.7332027304374554, -1.969331795235106, -1.6874396487391654, -1.9778468318512412, -1.4921133123175079, -2.301056376192703, -2.110583128505772, -2.0131150994308715],
[-0.47527894718925284, 0.06366771435042524, 0.1864963037910935, -0.22882339893292597, -0.06318929463198703, 0.26070787366500264, -0.2842948115878682, -0.3770241034324828, 0.025679443983239382, 0.0675884461802801, -0.024759263851301096, -0.21379943756479958, 0.33343456112992825, -0.293015787230964, -0.1380235093337016, 0.14341102300239447, -0.05868604318952187, -0.2717460065392681, 0.021965091488046644, 0.06624890638619034, -0.014717538783582302, -0.11684014168968881, 0.14831373972705533, -0.12543498318322474, -0.20073172214449658, -0.4215940637763204, -0.131473181781629, -0.32561635990664617, -0.5743460396804132, 0.06124222226375634, -0.4895220056168166, -0.22784539368352558, -0.46479554075038554, -0.7356534722080665, -0.6678046261325455, -0.8812739744771995, -1.001619276516235, -1.395273392610919, -2.3243897744025563, -1.8158871643978072, -1.2893836076996759, -1.239406879098076],
[-0.2528518405483855, -0.14857205630897835, -0.08281617513285172, -0.3502926289766647, -0.1501987969384308, 0.008840594663763216, -0.059579072776013854, -0.10723726363453824, -0.07114104195328225, -0.004181198786835399, -0.30015044738460445, -0.0798823398783381, 0.0979864929250663, -0.10828440331525059, 0.27425574401402847, 0.01012032394009197, -0.14751896373896944, -0.18691572050343142, -0.01775395535561585, -0.4055370762850024, 0.07922030119814374, -0.05699572900349496, 0.11897537694881057, -0.3700477308587561, -0.4212998278751892, -0.131226729358594, -0.03051983651165624, -0.3288903721861315, -0.6040703048598962, -0.16764202880277188, -0.74982306566887, 0.1402226682535819, -0.480334860806381, -0.6337634230644683, -0.3250977413467637, -0.5072627123879238, -0.3054846572312087, -0.979342070391669, -2.5074710077249316, -1.0476234773407433, -0.3581060445824069, -1.0891357269701087],
[-1.0971582838917968, 2.9488669415933084, 1.7509938650190953, -0.7660793069767701, -0.33981206818695436, -0.0318467078930586, -0.9888632397510043, -0.29667503749408464, 1.0947032429877264, 1.2184384672669857, -0.4611209782047395, -0.26696807233987435, 0.40693722722523235, -0.7990299762491282, -0.4809424448361797, 0.4578668524863575, 0.033720903326610284, -0.5578048902741112, -0.3429479556023107, -0.6344832760376198, -0.7533034810401029, 0.21813437331356372, -0.7106069537111693, -0.371194308336262, -0.1000656694757345, -0.5846576705632914, -0.24990623170088627, -0.11038306878277941, -0.6352974436981281, -1.6784436017169715, -0.7930198046925647, 0.22794292312088693, -0.7408159935728816, -1.1333797518202, -0.6890513529245834, -2.4247562773316087, -2.905898474113934, -2.9589309989654264, -3.1771713079674493, -2.4493123720415744, -2.42381719896396, -2.715167459588979],
[3.7649276082474357, -1.6506638554556001, -0.17433063699415263, 1.4147841998097557, -1.0257284796571458, 1.138552816386895, -0.6698419013289572, 1.5109070686516024, 0.0041219649864762484, -0.5885802225569865, -0.09726508232507444, -0.07646720745111507, 0.5000983953432605, -0.5997553079008737, -0.7172893078778241, 0.0592905188244658, -0.2978560542300753, -0.38697623521972263, -0.8201484037885227, 0.011120664933202563, -0.5873359062390695, -0.43502604129616507, -0.906245267216259, -0.2357839778215522, -0.946845003167076, 0.28266741664628897, -0.6596123570468546, -0.6263530647282489, -1.5259377931962694, -0.7271999654873925, -0.23358378235302169, -1.1110567907088338, -0.6908639727841452, -1.2340787051598088, -0.6290411237812434, -3.48724375173418, -2.911350435271411, -3.0572537726957894, -2.56027092969341, -3.1009041826357766, -2.67780837531625, -3.285305355365087],
[-0.008953647681650839, -0.005806764380813857, 0.0384788687200972, -0.22729684445123022, -0.0074123289380132774, -0.06034256699138538, 0.039518689818241734, -0.10537389749716394, -0.04251789766346256, -0.1810843035550149, -0.09169527401981718, -0.11441804767812173, -0.04677517878359427, -0.12250008803873444, -0.09877684581353796, -0.34181354647369677, -0.19676260282105265, -0.4068040319307872, -0.291966561241155, -0.43284690914522433, -0.1729692035980535, -0.09685929533788534, -0.0009235068943940609, -0.20101198284066468, -0.4739294529173017, -0.15944729113483017, 0.10393851583362254, -0.17183849670691592, -0.6583656446701952, -0.4224742483590228, -0.6524009262417199, -0.4545177928331309, -0.46329657837749333, -0.5404396652391568, -0.5293925796586155, -1.5254015762645075, -1.4932921802448607, -1.1858848242545865, -2.413035501081317, -1.5274976548252173, -1.348435973924578, -1.9393234060704208],
[-0.015987834950120773, 0.12638970420137774, -0.0615630308372475, 0.24847576033498409, 0.2167313256864595, 0.003048623283964219, -0.03989991186101945, 0.11308897368236981, -0.08498863798805226, -0.04999490460673423, -0.1449287258837997, 0.14259337549911255, -0.17307176446172645, -0.0732323984165833, 0.03573841210482732, -0.2278813293352349, 0.024410308654775358, -0.034673802659395785, 0.08804267351672156, 0.25517689616334055, 0.2348811089530273, -0.3942729141973747, -0.36302875668709483, -0.3452668657692724, -0.3478951299537202, -0.2729679192612595, -0.1644769489265675, -0.28955843310695095, -0.5782944012027389, -0.2895467935749884, -0.7947674808063592, -0.5147285463309123, -0.2552575463757773, -0.1861293697733156, -0.5036687279257216, -1.6533328358754777, -1.1137613413654397, -1.4300447455398702, -1.5935715987732668, -1.2445216801358212, -1.112560309899289, -1.1854430873956692],
[0.1773815906447209, 0.040302193115756106, -0.13804164824193552, 0.02946468733411025, -0.1835543478603739, -0.055462606168744356, 0.08286056773222275, 0.012242829069312755, 0.02339686997371652, -0.42761171165233425, -0.1270015610053137, -0.2160189661156781, -0.1127637713656404, -0.12120052679149544, -0.22501757702221076, -0.34816882998326726, -0.7191031605087229, 0.0609959692085146, -0.6859227009124083, -0.41602292316494044, -0.25621785590561924, -0.09788351298967045, -0.33976561446430137, -0.2466611724449831, -0.4524446354577727, -0.4067494294823991, -0.8853429546131819, -0.4215153913644345, 0.3863524177242776, -1.614040979756546, 0.803074930099336, 0.828444786888381, -0.9580739182103192, -1.1680368011880364, -0.7886138164861358, -1.5772782211416265, -0.7300958037095695, -1.5993481461428658, -1.7072065694730945, -2.191755836242702, 1.190065461150228, -2.335019149750172],
[0.01689001825002413, -0.16814712487067784, -0.4148182032023112, -0.30261656268099574, 0.15862096126857922, -0.0233683667295358, 0.4605329791432187, 0.1630845806545483, -0.30569249314770564, -0.5902462496068216, -0.2291872717087781, 0.4786896486479534, -0.28085082085566093, 0.11904736625337625, -0.07051967321097036, -0.41333045479512304, -0.2336635204106702, -0.09707574360827645, 0.4463900537512713, -0.06579723492947509, 0.09637243627441604, -0.11104264533323752, -0.45486925731935857, -0.5428361969883502, -0.36195318167325086, 0.09152053711104148, -0.5029117474308135, -0.3045481977381174, 0.011162248460661196, -0.513821527388818, -0.16279380918287026, -0.2998330271575705, 0.006653711478801781, -0.18111402247128378, -0.7519969119696265, -0.8322928042495586, -0.28439713011078926, -1.1114664967662267, -0.8512019783335298, -0.6401612746293036, -1.2350354597856708, -2.121591586319547],
[-0.028062374419221058, 0.10546550771596029, 0.1390901234545524, -0.05490545701382787, -0.10588454577298902, -0.2272769945898361, 0.05295747624107872, 0.07991807186999488, -0.12056163002312613, 0.04547566533423808, -0.032100839938119345, -0.08545136107256115, -0.20083405355322523, 0.04240169612225022, -0.23514527612650823, -0.17975716176812367, 0.009271471839331908, 0.0033908925699899107, -0.46781501884807025, -0.2749894848927034, -0.24047655795078396, -0.37027713888361297, -0.60217378232923, -0.42425535585213636, -0.46482621825292375, -0.7611902720710237, -0.8576229714050472, -0.5967629688106834, -0.795689259329464, -0.42457281642792194, -0.7427318324001067, -0.3702760820327091, -0.6736602033799661, -0.39844425485669566, -0.6783547782581428, -2.4405404812717006, -1.735280679393489, -2.236513908752501, -2.4968452976435067, -2.3890078535254258, -2.08924136153646, -2.400720743636935],
[-0.1987767055886416, -0.24941062391363394, -0.1820713115826418, -0.08394082142697493, -0.49515500026486586, 0.5593708412981288, 0.399408437156922, -0.06880681956377087, -0.23245173994962315, -0.46597298320166664, -0.2630792277231765, -0.05179389079290582, 0.4135850586900022, 0.43886426457837485, -0.31003648799401895, 0.15180943489016807, -0.3395210282575643, -0.7681133422211728, 0.03648866825789475, -0.041789550202708775, 0.15272110038598874, 0.051171109518103546, -0.35613406074635867, -0.025264889160017873, -0.3220844386804225, -0.09886673816490117, -0.40655961489147996, -0.1628702403932865, 0.39702728559981626, -0.596657874647591, 0.05913974983871785, -0.4682193317239991, -0.2515624719538694, -1.5072479074135903, -0.8424933416627672, -0.1309035178755594, -0.8476817287740963, -0.8178732308699217, -2.084214996067312, -1.7395417112230214, -1.3114503543497438, -1.615259148315187],
[0.3417358333621668, -0.43793982956655303, -0.3017609749146018, 0.11530974732329509, 0.3591070231179148, 0.05536133114561405, -0.25931202331804415, 0.2677727301928181, -0.3051675002054451, -0.276680842972824, -0.0036817564044082877, 0.2920906754725564, -0.3428319876749976, -0.15997860848974169, 0.12103588903399268, -0.238271097994857, -0.0548191886483274, -0.0034621474437391635, 0.14170321208996492, -0.010551899561024983, -0.24503243810064504, -0.3969790184264035, -0.5131523530587672, -0.30889363548928295, -0.2236112055357704, -0.3443322919069559, -0.3729778162049714, -0.37592479735023543, -0.20516749735731404, -0.3493516845888718, -0.34361423977979166, -0.6539846827457946, -0.8101241099699515, -0.20267152399407143, -0.7468489979027398, -2.453360791791181, -1.7445382904245264, -1.9981860361275612, -1.7658496158312382, -2.088262894024284, -1.7364351821141615, -2.0823432696235065],
[0.6051920626890576, -1.0480948733048034, 0.2017246169515252, -1.4005803189018926, -0.28122348113929624, 0.3007225226115654, -0.01779011506907949, -1.8305682853156475, -2.3980047328546372, 0.023770547247936927, -2.8595092058955616, -0.7816481546565454, 0.17550011021697626, -0.35389772533520436, 0.4203126639526983, -3.4219999258837355, 6.160886385736878, -0.12621651859982772, -0.1881209124251342, -3.0655364200929758, 0.22772828398442116, -0.8697498242688586, -2.719718868760815, 1.530443615828802, -1.3147812692412393, -0.5174001348495191, -3.6078322331696833, -0.907229778097852, -0.6371577449449487, 0.02671874063210968, 2.0178028817717517, -0.7180139419171426, 0.43005249837869375, 0.4448977925161614, -1.45432083780088, -0.22438624567315377, -1.4480480112136203, -0.2535230208445185, -2.9284388292691363, -0.5017940621110384, -2.195693766468227, -1.734194937493279],
[3.202225860918579, 5.83555974375786, -3.605141107129782, 0.4476945354854715, -0.18345746187720666, -3.0151389623755356, 2.411044825525223, 0.15536063636116043, 2.0923020887992907, -1.5135225202862639, -0.7131733073522276, -0.39259214655476643, -2.5889835826058247, 1.3487065368613587, 0.48985356552408016, 1.432528456318557, -1.2878871542485715, -1.1876776743776774, -0.982838433902817, -0.5757279760673578, 0.15288880147159695, -1.222164805926103, -1.9763308706601748, 0.14923821855660963, -0.2106459068338361, -0.4667796487304669, -1.6371673036848509, -0.8540449292494329, -2.5263529579102175, -2.611957301465281, 1.8661743864527522, -1.1214961485727655, -0.030236533563014645, 0.5983649455221273, 0.5436590918518149, -1.5178162777524034, -4.510723905128795, -3.249391890840144, 0.19009884193288334, -3.2172310668487993, -1.895869143053303, -1.3902479736588649],
[-0.4854169497424214, -0.2907885859600759, 0.008442540281278773, -0.6218143095434596, -0.15837930595113361, 0.6708384202361418, -0.1414032348216039, -0.26159859370020816, -0.0783903493919502, -0.3770165701788319, -0.28772341670798923, 0.1744963429013197, 0.3897380489089695, -0.1830012133844296, -0.11419172847045049, -0.171378020642974, -0.23041034524790097, -0.4398376345127165, 0.07145815981097764, -0.09743331148484395, -0.19969401676442208, -0.4031337298854122, 0.11526570179301465, -0.43705680989877066, -0.2703971024737473, -0.35435711409168175, -0.2794019101473995, -0.5131625465803508, -0.6810752905080619, -0.12349897879240873, -0.7444016980881325, -0.236979373232363, -0.3397695476310235, -1.0253346901496485, -0.8457952016615066, -2.8153758723051077, -2.4272717158552766, -3.1994808796207623, -2.9240656088691472, -3.1113454918801153, -2.697604203907204, -3.0903289783952155],
[0.3211319923039462, -1.5978995410549277, 0.1150662394163089, 0.2415329450278854, -1.1582109794545987, 15.878094877160072, 1.47511121178312, -0.10612415349464373, -1.1288556900084188, 0.9212540210757676, -1.1572283639396865, 0.06703989604314976, 4.212164589463876, 0.9798645640414403, -0.021292177057128707, -1.2210210465449078, -0.7621366406332647, -1.2960904580214536, 1.4575576922479199, 1.2403832756356123, 1.0246706677456432, -1.3236751872128139, 1.1359244892199378, -0.8508335033738694, -2.1721017945697985, 0.8277604879208157, -0.3412946116297508, -0.07690793140623918, 0.6460898398224382, 2.6997929835016423, 1.3960784109671418, 0.30018757897051346, 0.5533003652154701, -3.0990640982236863, -1.5037759894832543, -0.18489635722547065, -0.45103981523115777, -1.901700537589423, 1.4228341251067762, -3.5960134403641986, -6.735162384627211, -0.5291310878979721],
[-0.11420844225181563, -0.4791406182799428, 0.799878985763397, 0.4669233985850197, 0.1864024361074052, -0.17630896409266159, 0.04683609737737884, -0.3919046591550765, -1.1224185230223134, 0.7719385428820917, -0.33298188235244713, -0.4923566345320361, -0.6869488052017809, -0.012663455178493526, -0.18871657280231915, -0.13877461236800018, -0.330583785653178, -0.045246891179150046, 0.10983520522682323, -0.6872186364322236, -0.3040621000270894, -0.9695097027493593, -0.4818262460414394, 0.7989339008267723, -0.24126607630769456, -0.7150861126335537, -0.23963690983942193, 0.16676094856018628, -1.4713714958861113, 0.42125574577826597, -0.08453854829246202, -0.2512553193792717, -1.366106396572933, 0.11454532157030725, -0.07457138103383017, -2.532283862289545, -1.283606239082872, -0.664908861866516, -1.3890947711404507, -1.3961406809475134, -0.2164565547398185, -1.254562834795605],
[0.20812040524006728, -0.3661253369239581, -0.9296763683962688, -0.13806016673007038, -0.28397919256681947, 0.5605156310815402, 0.390857300644061, 0.16535070038636437, -0.38638095783207366, -0.9904390191454349, -0.4322129090698699, -0.08969997035118446, 0.4493586247918769, 0.13586052876162144, -0.3908602280964605, -0.12863643512508321, -0.38061655442492814, -0.0914376089702946, -0.21238565166808793, 0.1488707242105485, -0.006227455203697551, -0.46106750325319795, -0.4905079549531228, -0.4758192615274964, -0.01728626138071551, -0.2911539562510066, -0.23101715199609674, -0.0984827663345044, 0.23175713321004712, -0.8920398053039728, 0.06681289407089638, -0.2512229072289864, -0.5059067699952217, -1.1751893340089716, -0.7439421981907555, -0.8886127038476366, -0.8976973014373117, -0.9762110214651168, -0.854759422514894, -1.6758061014347063, -1.521072524142408, -1.5768305205488962],
[0.06933843010686072, 0.08848619465368943, -0.027782017584054432, -0.26529236365085385, -0.19592380970774162, -0.2982964554675507, -0.1479061460611785, -0.17624923553183855, 0.18596133308434282, -0.12341564424266102, -0.016489647728285973, -0.23201393189472372, -0.02840315014021277, -0.0664572556933218, -0.25711162326565606, -0.12427706872901133, -0.10113359900971611, -0.394701753123266, -0.4647747707276236, -0.20591260643262305, -0.2744621425866012, 0.08815298648113788, -0.2975464951713699, -0.21393474260913264, -0.1704353850543879, -0.41202984950825083, -0.3618861578585821, -0.25762010713257966, -0.2148963012073618, -0.624691272539259, -0.39999423261660805, -0.09333883731335949, -0.22650178578408015, -0.6306426620651869, -0.5051603398303126, -1.0583449559126001, -0.5093569575074549, -0.5895660338491545, -0.8025722759677362, -0.6280964947478422, 0.0754648804347071, -0.9486603699070174],
[0.40006629433031077, -0.27838461053051333, -0.13716454123627042, -0.2403346591210388, -0.2785057391890633, -0.16547072824413597, 0.11997831486585224, -0.09531005076636467, -0.26332050165877163, 0.07711049347674766, -0.31546667090425656, -0.21905495164976588, -0.41820751007121987, 0.0007635629478651402, -0.09632992644023801, -0.46333509366737247, -0.029421685270935333, -0.4941423067380119, -0.07351317083619609, -0.5006262973460629, -0.030843435689543452, -0.41145133948265145, -0.48909768923177416, -0.23934125642404877, -0.4346267005431363, -0.11666950479867494, -0.6526267420912154, -0.20717094948498943, -0.5640224574032052, -0.5725151738343576, -0.38926765589739404, -0.4823505462402314, -0.24063042415659427, -0.6730602152267667, -0.3106673508510959, -1.6965552923688476, -0.8614250190490403, -0.661667375043997, -1.929417246862353, -0.514049488544689, -1.0630207328554284, -1.4825196581780282],
[0.08015528192109303, -0.0814759917270937, 0.028135477904299374, 0.2283569634342394, 0.1022750331806769, 0.04293476787234212, 0.013917948349052535, -0.07305581345793277, -0.052804390198586494, -0.323875215763721, -0.2704669328944854, -0.06146507645452246, -0.10722387930460679, 0.023722379051571443, -0.14215957527794848, -0.4473714356079949, -0.161290031809396, -0.30989268798783265, -0.20109810582944473, 0.028318985315921915, -0.20172246131934796, -0.218063585159297, -0.5531260367726074, -0.41151344585492977, -0.5486090027250775, 0.06747678272598437, -0.27094504327593744, -0.3263234815163878, -0.3897639767915259, -0.8190183405360532, -0.4970701138598662, -0.47748598347812493, -0.3023333014418654, -0.4784795928666103, -0.7424626866815498, -2.1731840466749404, -1.6282021602067251, -1.9883859431163224, -1.9230565966006528, -2.022910551464184, -1.7300401961557204, -1.9205768721842706],
[-0.020090136845584902, 0.012074403863646893, 0.062315125426697586, -0.1235405383971911, -0.22994302180057613, -0.16171333173840824, -0.14428043734636617, -0.20134432540428887, 0.08479613017803043, -0.07841298395141312, -0.24163249690733618, -0.04961845190421214, 0.03304749967976671, 0.022084327139819206, -0.011340524705528427, -0.09330924487444847, -0.0016475089980669331, -0.3710632499089209, -0.21626967925826585, -0.14649805809326324, -0.1506131571249697, -0.3553286170830298, -0.1503316018864559, 0.09118747868089887, -0.33612601288436006, -0.19473317281108274, -0.15503002540233157, -0.21990238544360097, -0.5103005864069974, -0.22101923473936394, -0.11646625111148246, -0.16246130455846802, -0.26365224361944034, -0.4511820147719811, -0.38890508122167583, -0.7772312663865373, -0.7021557718192463, 0.5343492316623515, -2.01513181690389, -0.5010671811801216, -0.4085021613899691, -0.6512226027879291],
[-0.04725994342488882, 0.036385499196725714, 0.01997286365170255, -0.4985383998821648, 0.02323438327930261, 0.1427187327366944, -0.04805272421628892, -0.450301821725267, 0.044129425923536164, -0.2451441644342898, -0.49936414810088453, -0.2027273040377944, -0.07840471632919427, 0.22561523936024513, 0.08148395320136997, 0.2895914964091133, -0.4197873679438127, -0.876732223597011, -0.20314307932249917, -0.10513537147588267, -0.007047493304466888, -0.4124209748753382, -0.027359198281357387, 0.28273638669074136, -0.7191553201826071, -0.62818620453419, -0.5131883396912811, -0.1294537234476594, -0.579465451315338, 0.1800840877357522, -0.6704142855935477, -0.5251136942421767, -0.5449564126223072, -0.7118197121639246, -0.7242132518352392, -0.7215970974442936, -0.6548512323842278, -1.03601952325424, -2.122892284610352, -1.687574054174749, -0.9768955016861399, -1.3751042883898739],
[0.10836716760741104, 0.10134902605587917, 0.18764882084457046, -0.5697747970413729, 0.13804209786915153, -0.05003318664666909, -0.04890548057326867, -0.1260548990283774, 0.09906185857426221, -0.019461861117684887, -0.30765490549819535, 0.1692226114477184, -0.1445940226761024, -0.15934581191835065, -0.0801710676670721, -0.08014408997330448, 0.16890787528338888, -0.23281204881021916, 0.07400253493285358, 0.040566192927615735, -0.04736282913196623, -0.22940018066396917, -0.33606230268372755, -0.25186935884801714, -0.45016904578901457, -0.008231963500450783, -0.14826058483877338, -0.37084929652437504, -0.7630014209801043, -0.6907443313226849, -0.4353448950871504, -0.1359525662339651, -0.6003099161810846, -0.4568487329030626, -0.6451133977367275, -2.4068088457085066, -2.062325754509267, -2.3077266160329306, -2.554001083037291, -2.117558489872852, -2.1062140615443616, -2.520483642104378],
[0.19615672096447667, -0.14497530087379493, 0.0035600711482192612, -0.442005461492972, 0.04698639742217385, 0.24292147464215522, -0.022331295489419353, -0.23548501358491872, -0.33931743839225037, -0.061430701824919234, -0.47696055023942724, 0.08079243422256402, 0.01869166643961031, -0.24256089631239652, -0.1893240699401945, 0.0007481064766160558, -0.35745383354139515, -0.48147586369269035, -0.16502073739789463, -0.07869164025618008, -0.39506543242295833, -0.3885199895271729, -0.37936186395183724, 0.059173624758963345, -0.7318073024217736, -0.33736840399599777, -0.0602743147204111, -0.18112509620962564, -0.10427822023492105, -0.3641867748745078, -0.17932634169481657, 0.13289737385880362, -0.3895731478148392, -0.737805612037719, -0.4236417449851737, -0.24638347951224032, -0.02418701313588499, 0.3234963500660435, -2.175968388276442, -1.3015086785757044, 0.09914832236297896, -0.7782818045467647],
[-0.11087852790487489, 0.04406683652319311, 0.13123035534165098, -0.04410780294632531, -0.12338552237820227, -0.11084106249854489, -0.07587763855983601, -0.15084478651929134, -0.13556967932949748, 0.013474932603818936, -0.19177230060651287, -0.09568778979262234, -0.22091816786991955, -0.020238404352230246, -0.034649846163409, 0.0180069302935321, -0.19270258757503161, -0.33946461806560624, -0.2887060616549752, -0.14594683589665539, -0.25634706105038246, -0.39255697004331114, -0.054474696624344746, -0.1490847231601107, -0.3036167761866691, -0.3111787840808683, 0.0680814630375453, 0.12705762781616925, -0.6406814627401025, -0.13045167874296415, -0.336476229082135, 0.023547094419207305, -0.4341582117344413, -0.23789297973347717, 0.09648060312619797, -0.40367731689642883, -0.745707301510531, -0.13380905340023716, -0.8808926175674943, -0.24521717377753596, -0.4613043964926156, -0.2554600376629233],
[-0.13596624545293856, -0.37240228441598805, 0.2171145359731681, -0.14070170818059427, 0.40661751774770044, 0.12930353862283464, -0.02875535693690299, 0.10840469837896737, 0.034274168420905866, -0.8767097558459613, -0.2957335666036015, 0.32797907193940395, 0.9140181643306547, -0.05555360109259599, -0.18948816787098507, -0.3916666578902981, 0.7676999347805379, -0.7172696077294596, 0.13321910076105772, 0.5138216658948468, -0.5635693853756863, -0.6241886373007259, 0.45629991388563396, -0.6373456139487543, 0.6648993408251894, 0.05940190675302537, 1.7272972812652514, -0.5727665914085658, -0.9263170629891693, -0.5728873222799683, -0.36941316929480916, 0.3079981863089511, -0.0035269651088409995, -0.18545412466266364, -1.5066425956718181, -1.5661935007570638, 0.08756810648117411, -1.150314843054875, 0.8805535485190481, -0.179584610535093, 1.1394605797098316, -2.3070735175563675],
[-0.021371566421534368, 0.013905755161235099, -0.17978704871274262, -0.24875625867904666, 0.06481553751130008, 0.028109200793122638, 0.0173533266600947, 0.01517254592885474, 0.010425738690264821, -0.3197505358509196, -0.14515694328814818, -0.006687229656639013, -0.09543302575195163, 0.07625721421293244, -0.23807910687918074, -0.2939076028329405, -0.14338631862583237, -0.18374946177685292, -0.10116477500340153, -0.33177449853936886, 0.0058453282088485985, -0.24276051529007311, -0.29317958008296924, -0.34462954003126706, -0.5939056228075401, -0.22726871669909443, -0.28944095857472324, -0.29958997215309574, -0.6206242464311303, -0.5584839310788768, -0.37972965718138674, -0.23851560694735305, -0.39576000870275324, -0.35445083543452016, -0.6727678326725164, -1.3448755376817203, -1.2480406191879054, -1.0240014137592843, -1.959096898676881, -1.3085235112039522, -1.2558140894174497, -1.727973116821733],
[0.039013969765345304, -0.7377570412488775, 0.6915190801421096, 0.04603743172106388, 0.015907460824867676, 0.2466048015811749, 0.8815073005763269, 0.3074728690634304, -0.9120378090884426, -0.033945025448113605, -0.5066972437292002, 0.21261123891867253, -0.2997104450699783, 0.3170380173310166, 0.34211165104765473, -1.826900001282943, -0.24340789781603162, 0.19677565872182903, -0.10461864076434538, -0.41171441623366783, -0.43167792363325996, -0.0017369231980309797, -2.8740611540098446, -0.21971180598752832, 0.6860371533541046, -0.2924743495506958, -1.1266248817256113, -0.5582822739253724, -0.8280290851343621, 0.8102552749405705, 0.15345056600022103, -1.2669956611843964, 0.4365119770664661, 1.3756571456902054, -0.9154770007831795, -1.567259248228504, 0.3469557796734618, -2.062410692460657, -1.0170869574866888, -0.4650803298889439, -1.946025664047504, -3.170133559222733],
[0.015190189024458404, 0.14587561093188894, -0.32105572535255, 0.18147020772644776, -0.08802524551685062, 0.12728926334006252, 0.008294056041835206, -0.010602691840883176, 0.23783372487771928, -0.34567506998210557, 0.018298382149544466, -0.20259973716291302, -0.03098054711681051, 0.052865938575601, -0.13705646107359412, 0.005859586093063076, -0.2793392289799668, 0.0023987148707919066, -0.19857897823672613, -0.11404119855363028, 0.03894004059019545, -0.23188475702924422, -0.21972932350301613, -0.3618839239730095, -0.3931339572034674, -0.3458071961642965, -0.31664285105507783, -0.32386425449941003, -0.4824213692104676, -0.4929449849172771, -0.1450007982901551, -0.424526900846695, -0.39098711656785573, -0.4918486747392897, -0.7146735695451321, -1.297531270380233, -1.4570228092291742, -1.756490366631406, -1.4891270240091743, -1.8046761698468776, -1.6929776955847347, -1.7901583613589416],
[-0.45721047189019043, -0.3472355070754812, -0.31599237247415773, -0.10001940946257666, -0.022007051788545312, 0.4701364083758199, 0.0317006529856945, 0.0263402142096009, -0.03927553559894454, -0.30221943142423013, -0.04485960555422678, 0.05211491886626002, 0.27008966049183164, -0.014120562577674254, -0.08197254990692146, 0.005932778678133667, -0.37669095222646737, -0.07994087424944077, 0.0876627792021754, -0.19286777480619094, 0.017608606391069394, -0.03713959508426063, -0.17638201477897805, 0.1627093755506006, -0.5517560801960424, -0.3457892516855633, -0.3721266385638272, -0.2967387820513027, -0.10825118603664055, -0.1977915328149155, -0.005827141140310346, -0.4156246054255255, -0.04392183159127367, -0.8440051632381373, -0.6801832392090461, -0.9376332021566314, -1.048035071924909, -1.6202093156472988, -1.019582545587699, -2.1382664062836034, -1.389051710591124, -1.8331720589964369],
[-0.0914385390801705, -0.1817972782148749, -0.17596616483162697, -0.22184138370467946, 0.08161202660980535, 0.026660437396352002, -0.0869109192417683, -0.13277723264475386, -0.16164597800827954, -0.009062501636178806, -0.04901585202282189, 0.06142262995513774, -0.14828623963344537, 0.0026511850808604923, -0.010249885510096021, -0.04261334624649008, -0.10537843671478214, -0.23544992377205554, -0.22382332877584601, -0.1707664249831556, -0.28802175295248617, -0.17981030669198664, -0.22611814161807667, -0.2787043111059058, -0.4393725846324645, -0.39729315817076694, -0.3401674788080098, -0.03076321520546085, -0.5163688793677349, -0.44706007786524327, -0.4252773065289593, -0.24232076832440771, -0.3456615567752032, -0.569953499900011, -0.305698958589096, -0.7377122652496775, -0.740799306161057, -0.7640143775613415, -1.6058062972980365, -1.1198513030733854, -1.0275975870678442, -1.2531647281918104],
[0.4808016179785692, -0.427794261234402, 0.34053863009423174, 0.4548754651754978, -0.15254796602269358, 0.0900133819860566, 0.020483491837805037, 0.1084128760982413, -0.36991064485510944, -0.014156198408098232, 0.11788523072368397, 0.09410010394976155, -0.0063276607888351705, -0.1282228342247098, 0.006292264381128863, -0.3553889440983901, -0.03408599125178845, -0.011772983751511692, -0.28423692702499365, -0.18794761121874662, -0.2906468723744229, -0.44128358748503255, -0.8386444732570085, -0.011884476563619292, -0.25180837928043115, -0.29136555193362756, -0.08558857297120218, -0.2570982575831674, -0.724714595124787, -0.16119367428640355, -0.48498087290281355, -0.5858224673661846, -0.22525770829347896, -0.3449225792495626, -0.5573902415422347, -1.7547404422350412, -1.349152954812004, -0.7654847174476826, -1.9289073304278606, -1.2970805963784005, -0.9180015487822345, -0.8327209067010436],
[0.5695206942808111, -0.04280607277357573, 0.02995132256434074, -0.4375854841872729, -0.43841103241621276, -0.2604701849330998, 0.0774067849288501, 0.244535587870221, 0.2236894496811294, -1.3213817806651444, -0.5261976899244905, 0.14516477060986147, 0.24775418217349682, 0.13672949099028187, -0.10352710865429741, -0.1466656814969944, 0.4872929114526348, -1.8775816882149647, -0.3051209590500854, -0.6096879113376878, -0.7191200707942071, -0.73662503862978, -0.731983042378223, 2.2376739237748393, -0.3920747843303132, -1.9883324388412948, -0.9339425348355144, -0.20503116882258293, 0.22266336651292293, -1.8820204425549367, 0.47536230468189256, 2.245275842641132, 0.323418424125792, -1.860099594550744, -0.9435880156601585, -2.191434769980248, 1.5131374515501026, -3.501418307709936, -3.2117455385670373, -2.1116325574058754, 0.6817625347449986, -1.3430080714551622],
[-0.3857153862051739, -0.7128244652385827, -1.8353100692525872, -0.6410419209441576, -0.6904276959321465, -0.6856878653846564, 0.9070572492403807, 0.19996836734313445, -0.8081966928640013, -2.364194049711614, -0.8843313907624031, 0.8100302268004045, -0.37687787860494415, 0.33084910596769557, -0.7744863564579398, 0.1094176549012135, -0.5217025494735057, 1.6223280947948684, -1.9293250410436766, 0.8166803453459046, -0.3998658148065243, -0.1231543554409715, 1.4768369230448148, 0.5490177270401724, -0.9842941048068721, 1.0151932513709554, -1.1374229698543572, -1.5243342668145987, -0.6905532877386664, 1.7139545368951, 1.4807750740168848, -0.5921247340913293, 0.2685775512224261, -2.4280271872251307, -2.168521874431425, -1.2030349368194926, 0.9604289560576008, 0.018034490566892082, -4.283854081641493, 0.8955658843692359, -1.0464736690791692, -2.1661096733209284],
[1.9906543543126667, 0.829543498878448, -2.5746496516847692, -1.1642160328429245, 1.6686290934268577, 2.2102925308991397, -2.150308671289353, 0.8388955807369748, 0.09704933460494031, -1.0853110035241715, -1.2225237570199483, 0.282518048193499, 2.064708172629873, -1.0421471775360522, 0.7564266841644761, -0.5085354653031593, -1.201448293339939, -1.2550903173298689, 0.2054592645278352, 3.0631801452208136, -0.6013399545787306, -0.6541528754737548, -0.7201417448417383, -0.7223436219408115, 0.9189762140789728, -1.4272617755134633, 2.4861692369607074, 1.1458510452162702, -1.5285174062224605, -2.037185994730791, -0.04830463421190717, 0.2450912524941907, -2.4944532656892235, 1.3143211122907394, 0.24611002041970562, -1.8820888762031058, -1.5001976493325806, -1.2345888832608647, -2.793425554029928, -2.971273726221923, 0.5173400993451344, -1.8531626604824307],
[-0.08345843092505667, 0.171455024695179, -0.08138549584085927, -0.1545783636600558, -0.22282755526669537, -0.20124276193311194, -0.02662017550456014, 0.05838570770712476, -0.04275009973076641, 0.3773401498551246, -0.14366319927356502, 0.011383685780078978, -0.27716095165450616, 0.23390263801996386, -0.20618630788613448, -0.6354105450425422, -0.703541366661796, -0.1728471504513779, -0.3332852266092178, -0.30240556698509274, 0.20207507976244696, -0.48334690205207925, 0.301693394288246, -0.3241810914806961, -0.9746841817323441, -0.6016332929231286, -0.560014395021335, -0.1857346632606397, -0.61066998836741, 0.10820201361123138, -0.20317846868521058, -0.5518787469959557, -0.9255147029937233, -0.010097095966721988, -0.23425625765372338, -1.3046778918191715, 0.7734459810431745, 0.6624450550620716, -0.4727798157681209, -0.7674104525057723, -0.797612323480374, -0.30755109398969044],
[-0.959288885008894, -0.2492189223235499, -1.1173112379762724, 0.7497935564891238, 16.071364161255122, -1.1279196393449933, -1.593901028216088, -0.17302821718670403, 0.04131336839255339, -0.6512854067542432, -0.4063865006124096, 2.5827505810695097, -1.056880457772265, 0.7868014782476298, -1.114209192191814, -0.13284526592896748, -1.5560379220307685, -0.4518653276100855, 0.8696410665217256, 1.0026952802220415, 1.1208231897943726, 0.7740841746981771, 0.7562056705142486, 1.291823207402551, 0.5604251344729685, -0.3210373459714327, 1.0400548126558362, 2.165688096181959, 1.3530248566369687, 0.2526976625613932, 2.3866806427420064, 4.240269216918325, -3.3267732931212306, 1.19092460366137, -0.036373027317585474, -0.48559673544213716, -0.2421237791644053, -0.5748238175390523, -0.8681194107634268, -10.305107496631964, -1.4741789352748127, -0.9123367360690767],
[-0.22735357488853483, -0.22413465060905285, 0.25079460112497864, -0.2586811303008246, 0.3890376192468775, 0.27667359302919997, 0.45073835809653906, -0.05458555460085054, -0.141080479124903, 0.019342678676835784, -0.2672043144255885, 0.0036068596108485115, -0.14725999494626377, 0.3204948127526434, -0.25373379230042703, -0.31116152624565163, -0.08777005280613039, -0.5677635034908562, -0.4255197933455279, -0.08415412116283265, -0.029849039817293886, -0.03870973173435868, -0.36931820791608055, -0.012613049509179528, -0.1856949643049279, -0.5589299871462992, -0.5495429763519318, -0.19263239745361646, -0.19347518797312765, 0.2482224757526309, 0.1954928216679405, -0.34354681602467874, -0.339824491755661, -0.6790944001973394, -0.7719718436905695, -0.21535893294957492, -0.7618359512238535, -0.5714436495895906, -1.498042276482492, -0.8771202829961433, -1.0938776921310762, -1.552968211044439],
[3.367196188130604, -3.0713192906432467, -0.6470438605466902, -1.4055389517091046, 2.0492737160925847, -2.8707302401337946, 6.39876072626098, 0.449621899867458, -1.5161933563216854, 1.9454169790456741, 1.069009744366355, 1.9345876129123192, 0.13442915904141542, 2.9531062707406925, 1.344215916178681, -0.7802539294065663, 0.8477520992308033, 0.18634987427315833, 1.5499325399101624, 1.9336485891329582, 0.5529735989751011, -0.6480151827883756, 2.2097895615569514, -0.20973989463231552, 2.532766328201467, -0.6455167187191688, 0.4967419496566559, 1.1319840287087215, 0.08276364505828977, 4.359030541494639, 0.5155081097470766, 2.5719775115441728, -0.6517282271607523, 0.5871667319218272, -0.6637802579496512, -1.4259183779069007, 0.10590159231058263, -1.7816812243950482, 4.012588154309709, -1.3588494121402783, 0.2379618055916831, -1.8023626688320498],
[0.2770571468565011, -0.3081576326326514, -0.20630343787291563, 0.3520272390685758, 0.3521022297341024, 0.3733780327161306, 0.012463944045941876, 0.4080048947066557, 0.08606541736910922, -0.4346153080433083, 0.28821411802647984, 0.14033314571650124, 0.07935251121310813, -0.06657114953696412, -0.21374453433909324, -0.19190885017726772, 0.08230696495134016, 0.348683743470388, -0.027538497149163742, 0.07519096869709208, 0.02931038686009981, 0.32555218529539875, -0.9164891457298961, 0.33743137114674565, -0.41672805203419433, -0.32205924430518973, -0.3071399895449831, -0.16608151523844406, 0.45058313291766744, 0.12261938903898834, -0.01991227889242952, -0.6157140219197387, -0.4384538786284714, -0.6302790009359418, -0.530402142240904, -0.7695723665952259, -0.9655397895450403, -0.737693372688031, -1.0300438184906981, -1.5392142922348253, -1.2344963364088792, -1.4250397957311125],
[0.3290772335651188, -0.2884330851247599, -0.573602966609693, -0.1896852988422611, -0.5339994464438558, 0.7066636174775774, 1.016391722768983, -0.007582705642716467, -0.5225169269659015, -0.7866899719928191, -0.4472780902905664, -0.31811556709003547, 0.2176602279938657, 0.9626510025194873, -0.4591822126650903, -0.47374653003537054, -0.18114354496450658, -0.20819448341446445, -0.3733872816502722, -0.43178329413960265, -0.2710006762995004, 0.3865873977026362, -0.655253699259715, -0.22408729286402634, -0.17845706940148062, -0.6775133882958874, -0.87068213781101, -0.11175020440019523, 0.07365393546569135, -0.7985083125785662, 0.21643622031027313, 0.08252064557485753, 0.029914770721445598, -1.0362965227655194, -1.2811551598751199, -2.309142883790786, -1.7081957816146816, -2.1546365981171793, -1.7194347886857857, -2.1382053767716767, -1.916683938203929, -2.5399180343593324],
[-0.19855208252542292, -0.052273963106785594, -0.0287453333203596, -0.35884081740039314, 0.006111895804143239, 0.03980885864808387, -0.08566557798620253, -0.24289486397445073, 0.18261994647395438, 0.29078706950237937, -0.24703925081516817, -0.10522690870018188, 0.12252959883971644, 0.03775714858198314, 0.15061637566017785, 0.06611051424546752, 0.21246479242600394, -0.6114175079145039, 0.005970161323227438, -0.6189767999660892, -0.06671211549269114, -0.2680904541723982, 0.23011344598900083, 0.26515016789825496, -0.24213158217975267, 0.1229676839833055, 0.5217854707078505, 0.1792398357665428, -0.3700819668689773, -0.029643108529331955, 0.7311755395923001, 0.6011061070868026, -0.18079251578581268, -0.35376121315241316, 0.015732609257304464, 0.3078179708952878, 0.16200347378711918, 1.2101822094368968, -2.017783067411629, -0.5277841602115939, -0.15694270395837168, -0.4636120348235482],
[-0.13161973338011287, 0.002437537104807592, 0.12944781452210422, -0.25307264894505993, -0.17331948459435106, -0.10914142777810429, 0.10195516316352637, -0.1377105978518035, -0.022681909004809944, -0.23798344971084548, -0.20421540663278254, -0.12802429440580554, -0.0414134217012602, 0.0868400002770213, -0.16858597448152635, -0.20878111109061728, 0.0046749031005506305, -0.3693733190165063, -0.07052987248912218, -0.35503135933156477, -0.07928129557440604, -0.050847971247892976, -0.31371808856504846, -0.4014825604581272, -0.4812222455899189, -0.21032816746468183, 0.010816376436036354, -0.23167401793610087, -0.3555171419453735, -0.6155114321524244, -0.5119046210346798, 0.014062976095573198, -0.565760924356457, -0.7633485723096899, -0.43582988900375647, -1.5726925886578136, -1.0472419610427945, -1.3072708900215775, -2.106117649449635, -1.4137261658644091, -0.8477532708673539, -1.5808670890992842],
[0.26353224117692137, 0.17036123473381115, 0.11441202302218706, -0.0275960715647341, -0.3661977843344333, -0.11928436212013158, 0.1444388912579799, 0.06684176457444711, -0.3879870223150199, 0.06994326753937852, -0.1372453377931205, -0.13567534120241795, -0.6028332484055399, 0.22211079802790615, 0.419274492884585, -0.6721675287686523, -0.2026644640214636, -0.7385251527391045, -1.3428021209995102, -0.8789790940790381, -0.4056310215719652, -0.17093112456942564, -0.5510369078819679, -0.2703561702086935, -1.4442754023747473, -0.5223038332375877, -0.9838829040547632, -0.7527544699735249, 0.014564830005318755, -1.1603550627507173, -0.6534158949160963, 0.4854067369349712, 1.2152903485662347, -0.015532419562802578, -0.8410615569844194, -1.7945399411847367, -1.9383690249376804, 0.5103196820758077, 2.3576806521252056, -0.6759739543589763, -1.4768067327643786, -1.5768556528050541],
[-0.06144227218483136, -0.09677477820484062, -0.09545795097116569, -0.4956592579021257, 0.17413340952233147, 0.13001604438670972, -0.04232601297668838, 0.07839832147155976, -0.10362658218065778, -0.2903823721686411, -0.3047694204966265, -0.012907193116491892, 0.02057958184531786, -0.21983709570424448, 0.07782970425440586, -0.15636226298981737, -0.3140630114152157, -0.3998177866553472, 0.1293660945394934, -0.45620809383914335, -0.19323483670564523, -0.11163219484157752, -0.06075537498832347, -0.1431776285237627, -0.47638251437312173, 0.1274304555213062, -0.11306243516614377, -0.28052588926968935, -0.2698108610266253, -0.4759407904673038, -0.2644253895540197, 0.35444630068624183, -0.5871520281331951, -0.6591629419593295, -0.10223820100023862, -0.2619541122257859, -0.4210000932285468, 0.07743229778447164, -2.2217035726786327, -0.44151736563065025, 0.31496813867611484, -0.6768983615096776],
[-0.06299176057830036, 0.12989291124936936, -0.1532858869174723, -0.055957453794338224, -0.18262218836317262, -0.1399765352852429, 0.26714476369150736, -0.13168986020883952, -0.017972881311898233, -0.127243095067993, -0.09029283146935299, -0.3057461487449582, -0.1885891377438936, 0.09110713480101632, -0.5593850388337638, -0.003503492176350489, -0.6507770445407737, -0.17992050963301182, -0.7507964446677361, -0.3362903747471013, -0.2035233009272232, -0.1661243737396382, -0.3475061936656215, -0.2963094381783184, -0.5871598726592524, -0.23673974640639817, -0.2759977549920943, -0.12154142773687875, 0.7745609128681324, -0.8113039269545749, -0.10600543982384718, -0.3174909131540455, -0.20492789390045793, -0.9320322002108553, -0.463403903868056, -0.3427575068609813, -0.03059433297887732, -0.3558533328955178, -1.1159822719530377, 0.21626885867504278, 0.5118073564352762, -1.4099941457433454],
[-0.41311716169752244, 0.3507503016386861, -0.28427031790042107, 0.07858309460270789, 0.6328465651254155, -0.1538856482290801, -0.04233784807814457, 0.08779715584483536, -0.024577081835422537, -0.5834148102983938, -0.33121487102337005, 0.4049592159589439, -0.24191979385410958, -0.12421122227391777, 0.07983068835027367, -0.008192937523535495, -0.3968555272104134, -0.11149867859512451, 0.1116504021686005, -0.3531604527929212, -0.15868262049418563, -0.1923607685742563, -0.11866591883691845, -0.19964970053014652, -0.515032803778626, -0.16835058416775903, -0.6479258550905609, -0.02121104475277095, -1.0014853244340964, -0.17199124474957475, -0.44259174414939645, -0.7107951889659053, -0.6381449600651796, -0.5047795570185836, -0.903447325695733, -1.118929527470029, -1.4019667466829429, -2.25088777454474, -1.813057816588649, -2.241439931463063, -2.0244013635428013, -2.529757386311887],
[-0.1129078806481279, 0.003939676418440722, -0.25602548261222985, 0.28134214615334474, 0.21391890621593995, 0.030205625656974527, -0.05705499005172317, 0.07229005722461447, -0.242693125283447, -0.3014684778945115, 0.22320180828681, 0.006225832812442639, -0.06982821565589366, -0.19142003962525897, 0.07387699774990948, -0.45689723425621703, -0.31039023703187657, 0.08459478943554871, -0.07980717907231344, -0.09461281075515468, -0.2679620316329006, -0.3544271712418909, -0.7678502781018679, -0.2974956975375529, -0.23222758672426752, -0.45110331331342707, -0.2928797062772505, -0.16720261700842612, -0.19022524126925305, -0.029326353564337822, -0.5201526601446447, -0.3328395382614643, -0.5711339852662549, -0.4573696354481645, -0.7507648617774262, -1.6189392534078626, -1.415938033052878, -1.2405173335725506, -1.638208195456437, -1.6110094631079344, -1.3136388344142282, -1.6915020418018378],
[0.13443931667543949, -0.089734687134759, -0.08607813641015553, -0.13236818058062885, 0.013772255880657832, 0.006067211951600463, 0.12351016151708305, -0.1886013482547458, -0.17561090747021482, 0.006645485532794072, -0.24494245007599857, -0.0531322338453113, 0.007703918549310478, -0.07730131587557511, -0.22282029085657717, -0.16282292355388522, -0.19717080089233174, -0.28740551364004563, -0.2981779497334166, -0.196555152212125, -0.3021935716005294, -0.12183221404932394, -0.28014563988548635, -0.4926023057048675, -0.4205242745617519, -0.3579647971414209, -0.23761977617292732, -0.1468603272840955, -0.3811723247847887, -0.6061706569233701, -0.3366993954592311, -0.16154050083367152, -0.1845963170779827, -0.565550431186094, -0.4639495172528321, -1.2207651712502705, -0.8117144383412761, -0.9297406978671817, -1.1810689308197266, -1.2716999051836422, -0.7561340327311197, -1.1870886039420214],
[-0.6054571872559275, 0.45645312808154265, -0.34947673906969845, -0.35690936467132234, 0.045814091272892134, 0.1906772838387605, -0.08589086484640378, -0.34070453804450934, 0.5104427056088019, -0.581708582394711, -0.5753881488398446, -0.16960769024369993, 0.059692285851895684, -0.21254862215413392, 0.39452693724821764, 0.6477577272411469, -1.184195581540974, -1.0126755039916462, -0.5341469526991318, 0.09875728786544417, -0.6362411556453457, -0.6747416608977699, 0.048985844865373135, 0.6588424791364238, -1.1836429463341855, -0.2012648023859995, -0.32998346996651223, -0.09650044284153969, -0.20321738075751336, 0.4245118912322746, -0.017561252240139675, -1.0116954971149028, -1.3485688866972068, -0.48500186016087954, -0.7572633836574653, 0.6310926820400284, -0.3342792829088791, 0.940270443564777, -1.9536177334019098, 0.3296699953659755, -1.391671486783211, -1.514995778719324],
[-0.18083026738701455, -0.08135721269093618, -0.19015837846560243, 0.3527125106688191, 0.1506128375637092, -0.06554042996910525, 0.2629885418189026, -0.010354485168035565, 0.020026006690279253, -0.19286906655772323, 0.1417452699500346, 0.10207190621635015, -0.3085879281780352, -0.09471669653263945, -0.17745614858886155, 0.048830799519427794, 0.030901767362110602, 0.10490269579303349, -0.028857457248559936, -0.15780949292512264, -0.0671559268131074, -0.2946856101347843, -0.40176490688913175, -0.23648675307583591, -0.26750493349612764, -0.22153727873771206, -0.4771852736998775, -0.2354598601958906, -0.44948151311463347, -0.18056398531305384, -0.11924941013749427, -0.42414517479084607, -0.4183311596225235, -0.21890065668978903, -0.5155765891182139, -0.9108020696581993, -0.9509253717876325, -1.0806462987264729, -1.1806356097402355, -1.3264427547966473, -1.2614838714847425, -1.5926920821474628],
[0.08564203235947564, 0.21741138307040528, 0.10624020208258447, -0.16314753949544603, 0.05695637968206753, -0.12013177213838962, -0.05864031794113787, -0.22543011366942078, -0.03998424073245847, -0.047403178781777036, -0.14640965982725304, -0.18244379772059838, -0.08209033396445334, -0.16156746590223417, -0.13502088877189763, -0.25740991737535174, 0.03193586082247103, -0.7056011653391616, -0.2076124427896291, -0.045894243289432354, 0.2628163630469586, -0.20548521426651048, -0.043294058626613285, -0.028184325902668127, -0.5808531469816538, -0.12290826261683983, -0.22099696238172128, -0.005351131784790151, -0.6598733703984017, -0.19377634194953733, -0.6081223029845461, -0.36852328573493076, -0.46883636979091486, -0.4892664121083906, -0.19428399206374544, -1.576767962069466, -1.0849064165729039, -1.347836111036845, -1.9347631088727097, -1.1887002856591327, -1.2826941302426655, -1.2628548760721552],
[-0.06206771992228209, -0.2630813535008987, 0.06421403849420075, -0.29535416244993945, -0.03230183232625864, -0.05857513574771914, 0.15626059961846966, 0.15249154781165472, -0.1291485996885683, -0.2791449386568846, -0.19292768654982542, -0.092484228825905, -0.3248458059646149, 0.13577142548286694, -0.19906221121887596, -0.5572090196242253, 0.01770169559157187, -0.34773870300790594, -0.17922107091034284, -0.3445622297941516, -0.13716463596983536, -0.2865744190276232, -0.6677443029081092, -0.49724966816724203, -0.24325098759387917, -0.20042392531928754, -0.5844855618882637, -0.5044646522449023, -0.598272510326008, -0.4274565396730292, -0.47415299091494767, -0.49867366775627237, -0.47093380325763173, -0.6663139977484276, -0.7655444052679594, -2.1656688183270103, -1.6737723575898182, -1.7685203467372064, -1.9988438745998838, -2.004372335911663, -1.792929492851377, -2.3305000019252233],
[-0.18323466703282895, 0.19535961008679883, 0.059631065496586484, 0.11849266438920868, -0.6537839351035093, 0.3753456351940288, -0.02965442235880575, -0.019405512374899757, 0.2308172584427219, -0.28984039963551317, -0.11468363290786229, -0.3724693784569356, 0.29627998161326774, -0.10497462752812556, -0.17663690864735712, -0.07159229752510046, -0.1582936605956896, -0.05455732458646055, -0.37971836801426173, -0.013716154662242835, 0.030866739793699208, 0.0770285160293202, -0.18215055551630727, -0.4246350335011034, -0.3437819800650221, -0.3576819994688697, -0.1328946355388014, -0.023415917719799968, -0.702854258449191, -0.36875547869641073, -0.5646334024585744, -0.2320329490708817, -0.471802100698714, -0.6782527614547224, -0.688836512034411, -2.2282281240729493, -1.9207922101595603, -2.6244564535055455, -2.3676241540174017, -2.269395634456601, -2.1691264691877774, -2.2309124267290157],
[0.13157796890472787, 0.1178513877106604, 0.0763714986751439, -0.028009012839081738, -0.05049520780378166, -0.1448693701568168, 0.09581858149166465, -0.017364714184578554, -0.2766217171039434, -0.1714728673509684, -0.13042446814619435, 0.1520858381824665, -0.0639864427990759, -0.02331745245398628, -0.01588758953566028, -0.2155087243070684, -0.22945926711482642, -0.16721004236665984, -0.0056687452021307, -0.013794589215551427, -0.2302485975472365, -0.19317996370444734, -0.6278845142781815, -0.5420332933016951, -0.5513246296721586, -0.20458733866313575, -0.5152746489425197, -0.1475437385443203, -0.5363244306698686, -0.6626994155714707, -0.11829665918931773, -0.5381682115684443, -0.17587222879253567, -0.1652790134048512, -0.6870598647717121, -2.108306078848677, -1.4155959174384276, -1.7892672404561774, -1.4169326592792886, -1.3124143708153648, -1.3340874050251104, -1.856100317707502],
[-0.093000654736835, 0.03249400061125694, 0.018271288013746324, -0.0552564943502804, -0.09441558042387287, 0.045135865340607394, -0.14109777482948094, -0.08649670694864438, -0.02315083639549692, -1.1740764033507274, -0.8404225536154682, -1.14186245022032, 0.15168552697044344, -0.02710737399602929, -0.4608159180252748, -0.8627041703198347, -0.861022436882817, -1.1823431767328085, 0.6063887372965318, -0.10368670721753637, -0.02979633195601664, -0.8339544886637691, -1.8110271484105607, -0.38582683188159866, -0.5375280608664152, 0.5947645072912263, -0.5574181532220018, -0.5533447040893636, 1.1464537944278355, -2.775665569619277, 0.4470246479654123, 2.805277881198747, -0.2864500431679287, -2.1156292200602116, -0.9699651826243003, -2.8881008524411893, 1.630620871682172, -2.0940701821909773, -1.7452438858380335, -3.210920414822731, 0.2216730260707826, -2.429582160275121],
[0.09144208580472889, 1.2879529170588566, 7.885204585312613, 2.1837175589024134, -0.5123070454834026, 3.961740989262394, -7.724206365772103, 1.538028079849119, 1.6517091302897422, 1.321890755485581, 1.4243063632859605, 1.262444392562401, 0.30586075944841906, -3.70584976416614, 0.5274386168243173, 1.160511056711804, 0.5316665653506676, -1.4628164112331883, 0.1859588108234635, 1.4687536646823127, -3.66604631503083, 0.33331829937786245, 0.13879578648993876, -0.37594765068850683, -0.3337548679312324, -0.5594882216870336, -0.7492695197980194, -1.5423348496145046, -1.8636422157549077, -0.7791477747716645, -2.7162464836348383, 0.03411088818327038, -1.123157213230195, -2.0160280581663717, 4.031057979681031, -3.082526392255288, -1.3091320369433264, -4.804667860019284, -4.5506957019719545, -2.5202170726268887, -2.5043142155489018, 0.132922662856749],
[-0.11529312874781004, -0.02106151727692645, -0.0630542485612925, -0.03548030084066301, 0.25759664195822984, -0.034252922551194685, 0.13185295289590396, 0.006940802043094008, -0.09558922301100767, -0.062427529305857696, -0.20293229382833214, 0.14165197396684312, -0.14568880562792594, -0.2372454772969089, 0.09200915162971472, -0.20753518936553333, 0.05173648476085429, -0.2945547555003103, -0.14108854201523363, -0.13565327092825682, -0.07596371839517681, -0.2010855611228051, -0.294465508157326, -0.1054986636867309, -0.5558922326347348, -0.3975471868691819, -0.28524747243431886, -0.10550528593588189, -0.8062404057019353, -0.4436195723942499, -0.36442279324187704, -0.28006347512815893, -0.6399971788848933, -0.3885304720552108, -0.33220735440425486, -0.8726410751686312, -1.0248825046399306, -0.568845817575452, -1.3858272551153958, -1.0792287903568734, -1.1101428579864332, -1.072652260311318],
[0.012486643120852898, -0.7416546932461726, -0.23377662835505364, -0.0646064754497105, 0.14816929828266298, 0.3652085836225877, 0.35545131553476006, -0.051610386558377234, -0.22648798983656887, -0.1999412426360956, -0.17597785521274584, 0.1704359463940331, 0.41071796190554855, 0.28680532236102374, 0.07896199412815201, -0.34708934239706923, -0.45287230514811855, -0.16803456766973796, -0.1958297683901672, -0.06458936343998646, -0.2856271398817414, -0.14095585443351058, -0.9443348406275781, 0.3282625394674166, -0.30058359831085163, -0.243918529219384, -0.16610576540541047, -0.5201088047180091, 0.150420895473142, -0.00325905846193446, 0.14294391412136814, -0.17978532853514742, -0.45858108251924135, -0.9572411208172327, -0.8641553425172457, -0.5951588450220172, -0.7392272532408287, 0.1869514150105011, -2.1941872828168325, -1.2235474780561129, -0.7189283200933322, -1.692580891907325],
[0.2642431464530135, 0.15715011906845774, -0.24805884010124685, 0.06707414431980208, -0.04098654553331387, -0.027863330683363968, -0.05811539695132488, 0.11125331503720855, 0.1136497637523107, -0.014076604927510917, 0.08968584365755024, -0.2300966427482443, -0.10697145424552468, -0.0416008737489976, -0.3861356675369302, 0.2308731979554261, -0.3117128922253667, 0.07112551116066067, -0.35366440313961667, 0.006025420242409216, -0.1610807553027544, 0.032196328506799356, -0.5149464837295894, -0.20395140912492665, -0.5350213977593801, -0.5341794046895123, -0.3846406016271225, -0.20747298191845673, -0.12157225666807031, -0.4815199667459894, -0.22565647456616003, -0.6538630349448998, 0.029167320690950543, -0.36290665338522793, -0.5156664387709062, -1.183320668131729, -0.7765381373466287, -0.27611852057019565, -0.7300646932692524, -0.8076579696569638, -0.4480440887959321, -0.9300920619265158],
[0.3020511529409379, -2.4522547292252153, 1.4339344607652953, 0.4978316176899049, 1.7927548833999183, 1.5546719528734467, 0.43646242673406827, -0.3077763619374723, -1.108782817669607, -0.1736751703532433, 1.755923327042088, 1.1919075487035793, 0.4824363957884464, -0.2145145562046454, 0.31904387765209785, -1.9453447507315411, -0.19372917068027226, 0.009538904797805198, 0.7240615381465322, 0.6123427012018081, 0.4290950309933437, 0.3692872777912944, -1.0864339702278398, -0.6768463210025752, 0.26143317995453125, -0.500339358061049, -0.21648136840464477, -0.30358529562399467, -0.8682822942634583, 0.7338129568078848, -1.8845399920891093, 0.7952366910720812, -0.3352911503540022, -0.7371867105537452, -0.7048351398873837, -2.8844173314078145, -2.190507881446088, -2.5130500077579208, -3.648501905509257, -2.59148475159028, -2.651734876645487, -2.4179677844708567],
[0.26876454406127775, -0.22140952932278118, 0.7638209799508398, 0.343384739192434, -0.12162630481998658, 0.3624448921289717, -0.44635220515755997, 0.7574585974642477, -0.3288257054107555, 0.5633394604086763, 0.13302365755561854, -0.12137003338621102, 0.08846773128255386, -0.08514143038898583, 0.04008449402389848, 0.19097539867571392, 0.12284649718349913, -0.2839208318291819, -0.10542116554537036, -0.045805308734836014, 0.08625161464073894, 0.05497761682832928, -0.646800729147292, -0.19394197637110644, -0.6632190718765614, -0.25987517566412954, -0.2403352993406319, -0.17153156352127458, -0.5655706656612144, -0.3234097269457375, -0.5448217735592311, -0.7218255463640983, -0.4026503185040648, -0.06015430903074007, -0.42460458711359317, -1.050497269345016, -0.9329683627455747, -0.8142262136512867, -1.4513165993676638, -1.0804034429716054, -0.757970559224337, -1.2689500994730951],
[-0.21627799109654983, 0.058305575632079484, -0.214797619647895, -0.2555359375714624, -0.07972310075964494, -0.10496121044833305, -0.0068090746126488804, -0.1818835855756797, -0.1025026374629591, -0.17787386866425692, -0.25090180753808383, -0.007633151235266683, -0.19955908046130683, 0.013333017224358297, -0.050254085202667144, -0.37476858615994246, -0.08914131380782923, -0.21061093555309188, -0.24901037177675464, -0.12331649190967359, -0.27128488303940484, -0.10751759253156916, 0.27224049730198335, -0.09526840079643807, -0.5301847527430219, 0.0402927918313123, -0.4374142857189225, 0.05385765054530984, -0.4356228593383425, -0.3830105559046782, -0.33362797163104935, -0.10617845517520186, -0.42604064220741017, -0.3641165602604034, 0.21359751624646323, -1.0773758452457083, 0.8808398963865298, -0.1258222772952857, -0.9182487270206224, 0.18969795651367843, -0.8773849030407401, -0.42212948282857804],
[0.024192223743334067, -0.16007129973209072, -0.25624686573229105, -0.24729713845742643, 0.006658596691868733, -0.1160100649942137, 0.1574208585595382, 0.06315625944252995, 0.01422322331365837, -0.33941747782094644, -0.048497979321766874, -0.12460774055614879, -0.002355754020905337, -0.1249870389009296, -0.6535422938538448, -0.2529011081563136, -0.476329984963083, -0.34657568059836563, -0.0661154864347268, -0.43895775495709005, -0.03710812207560746, 0.3564246234030685, -0.6543713046954305, 0.009418458179513922, -0.39932446923206744, -0.06669105686848123, -0.2095600381177494, -0.17688605343012215, 0.7268820120515189, -0.3032321820670354, -0.6424559943965795, -0.4024398729630117, -0.7504765281163986, -0.465420725500189, -0.5605751415336334, -0.14192665589436668, -0.7969875799352294, -1.0285835884934744, -1.4005250630257482, -0.8684968277811964, -0.04799232190477637, -1.374296312529081],
[0.06561908100746765, 0.1316311608600849, -0.4617423111972627, 0.033985903342610216, -0.016412109898656838, 0.05016601946247678, 0.06072566208234789, 0.18597977060091003, 0.06752498144911391, -0.4899974706937035, -0.2331417794703438, -0.164598390361065, -0.08817053757587691, 0.042522956778847536, -0.23051248653976697, -0.20884285825826454, -0.6982814593129977, -0.38057720230748504, -0.43303306995626056, -0.19655539825191345, -0.20094665292660208, 0.09713067727849634, -0.3234179646710646, -0.4559835960615893, -0.2571111240955699, -0.5677561573124276, -0.289084560820549, -0.19729629024507908, -0.2519333348102948, -0.8488866370304589, -0.32272702062170305, -0.49420714397718407, -0.518772990567031, -0.5530280417572206, -0.551149340111898, -1.9208387902248278, -1.4872567388323212, -1.5250735871252175, -1.5045092028737765, -1.7156997445951088, -1.5754245211447275, -1.6716708711313126],
[-0.06602327337711698, -0.2534806273531349, 0.21192984090541833, 0.4662772561993441, -0.06530454405084643, 0.24129005730041514, -0.04207112036876581, -0.05837368596324695, -0.21193122082897467, -0.0570590359770316, 0.07250594981058756, -0.2411657726912715, 0.3171822931468101, -0.1758855685534693, -0.03825053812010398, -0.21191891405017313, -0.17853100791314147, 0.1326943448398987, -0.3099312079873339, -0.16268267005293302, -0.1163793398578545, -0.17941897078883193, -0.4029384917479725, -0.3491903522705163, -0.059135875049811944, -0.44654221624177964, -0.3017839063886506, -0.40949272689770255, -0.691513985388484, -0.25979223014118874, -0.5262695809854422, -0.37173663098882137, -0.3987139731172355, -0.4902951590163817, -0.6594036229166189, -2.777933977316168, -2.4462640597302197, -2.8284088414796162, -2.7279698069218186, -2.873521901617778, -2.774159079436968, -2.8683967155008303],
[0.1129207123650427, -0.10259971919130267, 0.17914646256780956, -0.08915551458999534, -0.04559765466610918, 0.0544687302918998, 0.28548277064952404, 0.22430891043782777, -0.11940678746740752, -0.08057043701909722, -0.3124482249767151, -0.015579385910517987, -0.32093996944546166, 0.15827262837701117, 0.054134396900793184, -0.11545948728341471, -0.23535129369304605, -0.44246246006038914, -0.053044373708314774, -0.2847088190658418, -0.16322393499165525, -0.5477398634277658, -0.4643931246334993, -0.18684521416484834, -0.6257552741674336, -0.3181362977542995, -0.7079397768423359, -0.37667444123352356, -0.8863576722339014, -0.42743784892303655, -0.6651662075031323, -0.5116863830285299, -0.6071147090419905, -0.32071238671543667, -0.5617240199113579, -2.787654313434452, -2.1856462053829837, -2.7439099403901497, -2.9013034630585093, -2.716435518649575, -2.509702607178054, -2.8802270233667913],
[0.013315352064383886, 0.07543210669046552, -0.19098767993652407, -0.04324356811463036, -0.024922291263937177, -0.11751502243106557, -0.10437829755622723, 0.15668902954373867, 0.07551656742086756, 0.0027313232306407412, -0.344421007179513, 0.17244942432680974, -0.2071609284643225, -0.0808838320831152, -0.2752102267683253, -0.27022797142981503, -0.3049267359094769, -0.4598306422216979, -0.11027710141301857, -0.43896436856128096, -0.24112478456160108, 0.04734507234847006, -0.5033818746000257, -0.4736115994244247, -0.4186769967608233, -0.17311640060571276, -0.32286331513527067, -0.15756071069941047, -0.23437739687736947, -0.7337472937097599, -0.38261758147007896, -0.1573843197467137, -0.27714691716364515, -0.9048983146875447, -0.5451530667319398, -1.8773063574934628, -1.3334702191846735, -1.6649979916756454, -1.8319924006384198, -1.612327483141501, -1.395232978145396, -1.7061134888106761],
[0.033553379950039486, -0.0157432806386093, 0.24433150345296475, 0.24330237015585365, -0.2568011755351491, 0.5489717033740058, -0.2769964509068257, 0.2701294109805769, -0.08655099153604819, 0.11687313159810064, -0.1719515367020153, -0.019485104228556573, 0.5293910545783603, -0.2286079053847341, -0.049649740447610155, -0.09329964561073087, -0.14067660527829853, -0.28822196125872923, -0.5000317594312953, 0.09573626584557514, 0.05332936993036564, -0.018934101266473363, -0.31917125838047467, -0.17307506842872447, 0.029098106661768965, -0.7721133369342261, 0.06993434881507252, -0.17966830770299916, -0.8038633818815067, -0.6274129321465566, -0.6901156937649122, -0.4590973358702765, -0.6409326548454203, -0.5222489089047876, -0.7045133006230323, -2.1971811515976283, -2.0211772958027767, -2.142710894269146, -2.494872579002953, -2.5148155278756126, -2.154768144400963, -1.9683584693305216],
[-0.05462417187667894, -0.19170004951632008, 0.26106892104857776, -0.14147390674755933, 0.13838735794490914, -0.2946194622242232, 0.04417902197312572, -0.07101508316015086, 0.07519597541906328, 0.19126243242071797, -0.28952264989754783, -0.15480846121278471, -0.22493611377601389, -0.04646244364190904, -0.20365110981858955, -0.11904677099391406, 0.07221064306513379, -0.2073886592003876, -0.22096630670577386, -0.031240348600645593, -0.16065195380592479, -0.26626434985097114, -0.3574794538253473, -0.174327878508239, -0.45378356000813297, -0.08344147274668105, -0.2566711124828069, -0.3628273687421782, -0.7997224841268992, -0.20619415078569686, -0.7253388936805494, -0.45189867107766524, -0.40570391437197184, -0.0940091638210684, -0.5866644625648861, -1.8337726474924356, -1.454232721130518, -1.5041868711501838, -2.231207271754233, -1.6208673407123833, -1.4324393994726143, -1.8596988282433267],
[0.27119784166887084, 0.31454115219595974, -0.04435771002064374, 0.16142884336245358, -0.15344727981125592, 0.431762042248073, -0.12253261707881423, 0.1639362174981054, -0.31967292177548584, -0.48625107372399545, -0.12868531815647516, -0.06929331527420854, 0.010143996452053551, -0.11809189507227014, -0.14237307931108276, 0.03422158382563074, -0.32013832867968867, 0.26079162417658724, -0.5500861084310376, -0.5179244552619353, 0.0255174066243784, -0.3665270637295454, -0.3410779377351099, -0.07574647824563674, 0.007189055506228017, -0.4858584571570792, -0.3169392995153039, -0.4905422096832255, -0.6657528698376624, -0.5177800700186003, -0.43908984969552595, -0.23678036512280942, -0.8519907661277456, -0.4971153713184848, -0.8184856210434261, -1.9631396565211072, -1.9615495939336476, -2.0492100301349536, -2.3460811729256, -2.0948615311731063, -1.8005850926059639, -2.1017319318431413],
[5.007424546297214, 1.1378002793861763, 0.9483207589295194, 2.2294164405978374, -2.454505267191864, -1.3525943623702836, 3.2111067622242166, 2.203574544325705, 1.5771970635650485, -0.8087640990140527, -0.21216249971688703, -1.462743760172417, -1.1508076236734497, 2.0948495291321203, -0.942967149374593, 0.3108068962853128, 3.001037561500823, 1.4284752317476574, -3.930996321508056, -1.5735836850319977, 0.3470543506524054, -0.34715883630466254, -1.7108630052685487, 2.137970840951424, 0.6186289665954944, -2.9269059847538594, -0.8214137424171388, 1.2389248459257811, -2.200595311825784, -2.033597299195871, 0.9706761529923807, 3.851429642156681, 0.8319245428696098, 1.7848941708396253, -1.2059712582948685, -4.407948463469599, -1.2824102805777615, -2.6477974596864087, -1.7891972299605934, 2.1903623513945516, -0.20498530645400784, -2.7582676171786753],
[-0.03900597276791964, 0.3934460388889435, -0.22457849555495868, 0.3475765405677595, 0.3629004012942838, 0.10707480009889739, -0.0150714964082761, -0.24345950792459597, -0.14110798556715023, 0.02973832244152749, 0.15830807985872397, -0.09372136709503992, -0.19342475187771238, -0.010850242029670092, -0.12154827666375403, 0.2224476581709509, -0.3777071131988879, 0.3444490547431898, -0.30183368683550876, -0.1669565668578093, 0.0046898787216978636, -0.43968467272610884, -0.05346043640899888, -0.6131968656561089, -0.10208149309893454, -0.7753769406496043, -0.5939989218475091, -0.01300044652495138, -0.8008443050139912, -0.22511258693942462, -0.5282995882134467, -0.7852373700410618, -0.17160262111028826, -0.16778374080497624, -0.5627922703308298, -1.2477143726264999, -1.7785405356515858, -2.3439972597315064, -1.663207604356712, -2.3906295692977784, -1.9155936597235526, -2.294548046743115],
[0.2012421801713787, 0.3793321639748062, 0.44317937856389134, -0.5079901378442467, 0.1041453429599128, -0.22304386210467755, -0.20718440172939892, 0.30405006263470025, 0.07294232353425782, 0.29779533329472113, -0.0810093927068422, -0.20826701745234877, -0.6226295020354135, -0.28485081439420007, 0.3101033796013728, -0.3190914079490786, 0.04987993997831377, -0.08169839473179304, -0.33240573321079064, -0.47953171823921437, -0.1561066573071576, -0.3993358727377927, -0.6482820542673127, -0.2565750650711008, -0.7157803821065915, -0.16652104957944486, -0.50303418607936, -0.17297416139422916, -0.9358163971330045, -1.0105588383511874, -0.5410859691954321, 0.0027254461840726077, -0.0843012980974312, -0.8255567774298184, 0.4354835821703398, -1.5459760826750713, -1.2639139538190298, -1.3571115027963714, -2.5853682065709958, -0.4451996455473258, -1.2565699706897058, -0.7057532109849916],
[0.8628022531360882, -0.797166856596298, 1.285457485536845, 0.27611659962785534, 0.21036261259097314, -0.11317770567679694, 0.41371512759157897, -0.31509090306994775, -1.139920463244001, -0.21593703828249025, -0.7537248015721851, -0.3404843561113741, 0.07077940494393761, 0.9209294748889465, -0.08048162757580471, -0.631651758806995, -0.4136959695743894, -0.9532933459955671, -0.42428536084141105, -0.2265639086742915, -0.03959637043995115, -0.5447719930622971, -0.4264483736490988, -0.8373290746745137, -0.6910945545270987, -0.7939980930144176, -0.3267270912043655, -0.09566456865650556, -0.9509901107825144, -0.2980986916473858, -0.2632159190820035, -0.6480397495971632, -0.49068982147893053, -0.5252215889089024, -1.1223162390665358, -4.618720490817906, -4.17432789047996, -4.591781418862662, -4.436214146585433, -4.576945091985951, -4.354413443261565, -4.614515236823108],
[-0.06648074698622718, 0.38674965781731263, -0.17857522009573745, 0.8912997173413937, 0.3659606660867342, -0.032651412130459787, -0.24529449140866186, -0.8151008702991599, -0.3405261954926507, -0.14950795995210742, 0.46317895372396006, -0.10045718790590678, 0.007400258254301878, -1.0836907039080275, -0.21559593071373542, 0.4074994191503367, -1.1717277367096357, 2.760515285049101, -0.631043277923003, -0.30766710646299006, -0.11851897484563584, -0.35832488517762595, 0.03907740212233239, -0.6383479445972711, 1.115275339560608, -1.4075191957878628, 0.23404432320798446, 0.06226422423263816, -1.1854865264198853, -0.35972959842016894, -1.7562410964480617, -1.2342818232898067, -1.5496599664555675, 0.4579042719480672, -0.6320404621700106, 0.15559278163941692, -1.6067742572753039, -2.4140593918277387, -0.08851335369253166, -3.105693526394101, -1.8910002825955647, -0.9844791362217188],
[-0.24573464936974496, -0.06929168678862208, 0.13321987920656728, 0.251094843606843, -0.3530437100029342, 0.012344865709002254, -0.015580318014138905, -0.017349438313696355, -0.15276744028094802, -0.010532363385171396, 0.2183648888034152, -0.3370063871249864, -0.11849669563721699, 0.02399320789276655, -0.25434944919856045, -0.016593460331293462, -0.10814719856949925, -0.18124036151436368, -0.3838002224772864, -0.14570538507340391, -0.3079849133718712, -0.18289867702739415, -0.26026788288823455, -0.07861844330159468, 0.038740032836670026, -0.8805571301171052, -0.3808159214177543, 0.20347876021961778, -0.71959300246415, -0.537013126222612, -0.5511913921929606, -0.286820293772603, -0.2035945332591219, -0.7338992915813713, -0.6050920906371757, -1.8965782175624268, -1.620032599929442, -1.8972278929840836, -2.0306069249749807, -1.6315384888166897, -1.657767954452094, -1.0934164407393303],
[-0.22944349259501373, 0.010749066345966889, -0.2505041826455357, 0.3276536978846833, -0.07255626515961099, -0.13961346643369923, 0.30431529182115713, -0.05545425288307845, 0.005040390379735332, -0.17020625312677481, -0.04737777957720759, -0.25370527254981484, -0.24421853468621335, 0.026659554913590164, -0.18734523130639325, -0.018422773094307357, -0.216674915031965, 0.012611354513526511, -0.08103627010003922, -0.20972497490773773, -0.0009826705161150345, -0.13221619426454956, -0.31001340860386917, -0.36495716897607267, -0.08540142099189038, -0.4672133796328, -0.5234704776415167, -0.14683624649855598, -0.45402218444979936, -0.44485949187589924, -0.2062870319333241, -0.6686199637007256, -0.24259101591155, -0.3418303562199564, -0.7370116757618712, -1.173675867242761, -1.1661564944795635, -1.6820796406984755, -0.8945292957084473, -1.6314517863475688, -1.5827709120067202, -1.5041689733592396],
[-0.19502965365077887, -0.19607639397839316, 0.07575174992062059, 0.11520261343705963, 0.41157535759708147, 0.05869875180959613, -0.22101045519109816, -0.12513530136671108, -0.01766783335216836, -0.08376594283219363, 0.0011267131119123506, 0.15077632425058116, 0.1047040601557241, -0.025514939567324188, -0.11602807696800721, -0.2127290452205815, -0.04752618203842148, -0.14083299690270265, 0.18867052007160365, -0.14399134378920175, -0.12228791343797825, -0.13234656681436816, -0.2727846791191608, -0.18932175270059554, -0.1931647818066485, -0.20767448721849086, -0.24430120964386515, -0.2267091913535759, -0.7206719912143257, -0.33325550684797645, -0.4334751152299935, -0.35599113865774307, -0.5651194273527737, -0.5992128667609978, -0.7580570016450076, -2.326555633399306, -2.074900061582975, -2.2683363175312956, -2.5472714775121923, -2.624469215777978, -2.26181218275211, -2.559614590578928],
[-0.1514337815048371, -0.15385471295140324, 0.1353648907303359, -0.01963619325533793, -0.12202915858788517, 0.14010187079470823, -0.1299525817661324, -0.14603548177982972, 0.02115890148704811, 0.07661072495822831, -0.1359120840142728, 0.04777489630232258, 0.021973437341516924, 0.019259185753855785, -0.12101513579278382, -0.16455301481346424, -0.010760383112406147, -0.17187769237038322, 0.1356742654905689, -0.03035367168260469, 0.06646647571014484, -0.02193409527137667, 0.10510359451785413, -0.07575942934235542, -0.29001046991817053, 0.159231612358982, -0.1926836260645701, -0.03449116614330932, -0.3442740264832085, 0.10213904831520366, -0.04870625278602184, -0.14218327806684297, -0.1312485293765658, -0.19482816299681396, -0.22622264301729209, -0.04371795139155349, 0.3208882457067953, -0.26862222636448707, -0.8389361469986906, -0.05274278223912705, -0.419100802309932, -0.6785811261862121],
[0.04731596742347376, -0.07177239916567839, -0.28471879443701, -0.29039943786960704, 0.19645683971352054, -0.04952220947291128, 0.07363969579907588, 0.012152887812459595, 0.02771011101077621, -0.4700861126156124, -0.11208474398486329, 0.012574015133638674, 0.13755161077658176, 0.016377864752158234, -0.27900935637399765, -0.476706523486407, -0.25993092822301356, -0.3714894564330011, 0.30894261576275034, -0.545033814569737, -0.22356401055701153, 0.02120648517341106, 0.028535455440230742, -0.6115649252897337, -0.4854209818637124, 0.2961878001134954, -0.14369802892895972, -0.25694828803733105, 0.5873399656027544, -0.7803517678002199, -0.08462199626146785, 0.15316883622425698, -0.23159103531351374, -0.7031957607703196, -0.6386864211708632, -0.2878000666491339, -0.2567125435735516, -0.7584638545081096, -0.706981448508539, -0.9371017130212909, 0.19423210869459298, -1.3806806235410565],
[-0.0763199869714117, -0.1531836133895463, -0.13717785022827705, -0.25385253097247457, -0.15395630139518035, 0.09243008058476658, -0.04314449611069035, 0.015828694440377034, -0.5053084196037155, -0.2266532815069923, -0.2622370574071765, -0.13555374979463863, 0.5868777385663126, -0.15537043022429856, -0.08594144427320037, -0.34726104550850573, -0.16537717728724746, -0.618853061100052, -0.8657103085243342, -0.8039896859317002, 0.29190068737274705, 0.060309877835466444, -0.5101281827044168, 0.39674462066530486, -0.7643944114384317, -0.5145828319412448, 1.207376382836901, 0.3573988503378626, -0.46101317703511496, -1.0839301208384726, -0.5180020704189956, -1.5407489643124772, -0.420369131768825, 1.532898517721333, 0.020514493111189636, -0.6928912849133505, -0.6172320074588162, -0.8546161303949267, -1.5267130967461349, -1.0937113666190932, 0.8914183801054101, -0.6754244619660552],
[0.39588002869450695, 0.12761290307626194, 0.39498845010822914, 0.2120132791637351, -0.2570162625653228, -0.10686153627564887, 0.12904049128550663, 0.022783215484538405, -0.32874649826316044, 0.045470241751498774, -0.1257022568242184, -0.1549763623848011, -0.24698643674031512, 0.06307380393615325, -0.4584798337635877, -0.0759014850778124, -0.31431073336033144, -0.1309150465548653, -0.6164508578912941, -0.07336933274641683, -0.150980464579105, -0.24488336393341775, -0.37255176871336826, -0.3079805070166726, -0.5691072453453562, -0.5464454610751694, -0.48440484984927684, -0.2100173417567483, -0.7281517791305381, -0.6238597594642089, -0.6882535037650356, -0.29887997987807613, -0.1358318972252168, -0.3547215239129954, -0.6233778248776269, -1.918259770317803, -1.226472576060433, -1.3075344513084255, -1.653946818055275, -1.0056576854685801, -1.1779947957121382, -1.0815347461758622],
[-0.2785213270106117, -0.2465310557007645, 0.16280777289195086, -0.49377965048641126, -0.32175936205036026, -0.29044832172661694, 0.05157422686123759, -0.20211461126824817, -0.2178149181809636, -0.05792523832752425, -0.4938973513241083, -0.12746961459476433, -0.29572998663215666, -0.36048475479137304, -0.4108869441601125, -0.3361687632620913, -0.7735960020492683, -0.5464662824257196, -0.873369163000062, -0.4752456452005285, -0.8377691202617433, -0.5334391850095583, -1.226831963835132, -0.8015912938902948, -0.6947970164923467, -0.8884684919322454, -0.8970942573971763, -1.012736442616473, -0.10323275925512235, -1.316526431810295, -0.3382130207654297, -0.24488122939392268, -0.2727288864239923, -0.27072183160386, 0.39636260201320606, 0.00829650518874637, -0.23322791170858667, 0.43318806871577914, -1.945474558221638, 1.6647412789449254, -0.1682271266217433, 2.0164763498527996],
[0.03726362581793572, -0.012173694222269441, 0.1411202028209187, -0.12078501161359334, 0.1497205641545915, -0.1495718130019357, 0.05297943593937543, -0.10460726917288167, 0.04980008993335996, 0.3143498896985015, 0.02888939323787707, 0.2728436489372209, 0.2486584798164413, 0.15735493899276656, 0.22332597553085187, -0.041139613606278415, 0.2974987130362116, 0.18332615564551727, 0.4017875639089339, 0.2873536536291762, 0.15038111421026232, 0.21594282328379374, 0.046135944841964184, 0.5317588602602658, 0.028106912625025633, 0.6169053164362812, 0.14345998147166605, 0.08491371959352248, -0.17029570518116308, 0.03656326219112273, 0.34381138974406394, 0.3560458639889768, 0.36574602250831645, 0.2779392495257934, 0.41819041255023137, -0.3903970917633384, 0.14200507799352854, 0.7526502400414581, -0.6673203264904958, 0.5734290414498766, 0.28394240833069845, 0.04929545485636467],
[-0.1252825378114915, 0.18180755100767743, 0.15501069671276588, -0.19112598382292156, -0.15405405421681284, 0.05928151992510914, -0.11748317531556582, 0.04954026141645146, -0.01773614807972782, -0.04955515315556205, -0.30692409808193283, -0.30621735313113685, -0.08994378017863232, 0.005836287575790963, -0.05709423945408989, -0.15946104701434713, 0.16828650855305172, -0.2648331095163811, -0.1683428388115476, -0.1760651069203801, 0.13286632647464156, -0.18749384261056054, -0.15282293266827243, -0.16193556415983984, -0.09589431768395805, -0.5100941464622536, 0.09436557589400119, -0.1778063358769029, -0.7767884580054886, -0.47485943208993814, -0.4966752008023782, 0.03635774847288228, -0.6835652680154584, -0.27715322096775274, -0.577846429709181, -1.118111567413638, -1.1297848016706271, -1.1596898354501555, -1.9015517507211985, -1.2609907934784146, -1.0874421288818334, -1.2210203116797906],
[0.04457484753064498, -0.08552150870759012, -0.013165453080043623, -0.08510973546801252, -0.3955658720413644, 0.19475293030300925, 0.09098066810591642, -0.17605117625508845, 0.1127747245476741, -0.01288623380662333, -0.2740850416527734, -0.0699291657227121, -0.023791101732094522, 0.07903170249954461, -0.07047042012858666, -0.022208887616059568, -0.31110040017315455, -0.3828250662838431, -0.29716921435082394, -0.04147827004473954, -0.1410955072971428, -0.10339575291653443, -0.19308770292265806, -0.28702978832182213, -0.5232236481246803, -0.24229606174360196, -0.3062356845782837, -0.29120455881231944, -0.1940498922667173, -0.4920042330403995, -0.2784787495519133, -0.17184861592496206, -0.04312192139036559, -0.7030558012095945, -0.07987873915644221, -1.5571413536389005, -0.5008865999616269, -1.3593360513299446, -1.6805172084950815, -0.8906013340010198, -0.8486336192358712, -0.3980968922623626],
[0.12594780260697933, -0.6691225491559548, 0.2480467547659914, 0.3115811727032356, -0.39900151040327136, -0.006585013420969014, 0.5172382221360059, -0.07650109369026022, -0.46246609259342397, 0.13818959370622688, 0.002055161133631324, -0.3318732525238558, -0.1379581997758873, 0.005033776036284902, -0.056031466772978634, -0.20951473523664396, 0.23616929077587837, -0.12654469114908515, -0.20364146912369904, -0.15471730165243233, -0.002490065778774257, -0.21696500874666586, -0.6114585252118243, -0.07823565042398238, -0.2464523629633121, -0.21554377138223355, -0.32684423807067275, -0.38397788436787456, -0.7065141476147664, -0.07128403455628635, -0.14626707324053156, -0.38459845875165954, -0.3980425612393984, -0.5724385383316726, -0.7099503920621659, -1.9913894202952476, -1.7701542657525717, -1.0248755292290639, -2.371024716494502, -2.0338278702390986, -1.834165675088437, -2.1598543864281816],
[-0.08854320552344346, -0.29213352298900397, -0.273319467784778, -0.491751782643087, 0.24388983140543002, 0.31880881926863597, 0.3266026882115597, -0.2168936985404135, -0.2513295306738096, -0.4166803388139038, -0.471759081810698, 0.06234444445939831, 0.2048143684615497, 0.27289125459589175, -0.10642593805477144, -0.5485365960873202, -0.18200514107675514, -0.3833587018528916, 0.5043440381623592, -0.3933454770762224, -0.19384432089704742, -0.26584876849724925, -0.5011372346901288, 0.12755856211971436, -0.3720404767887388, -0.025091765443681992, -0.19821750320139295, -0.08113878845179341, 0.032418612833415364, -0.6552614167675268, -0.5092735760900841, 0.40958159640851943, -0.7063003279226173, -1.0890385760173045, -0.293711904656741, 0.05424746929781395, -0.35549081785170855, -0.8264281838125499, -2.5164773851957896, -0.8807919261011636, -0.046364669566779544, -1.1365787003949963],
[0.16518790495583868, -0.3298847049160798, -0.2674278993314386, 0.17722508067456239, 0.15458237206963657, -0.16132670714952704, -0.30469929246517774, 0.2231858642542957, -0.31511015030465095, -0.5044904614144345, -0.1369247698953707, 0.046302611540003105, -0.521015401750923, -0.5454510774642615, 0.0339924928516999, -0.44002829030570506, -0.08204105387206309, -0.012707488124838805, 0.6222791191454633, -0.352056460108305, -0.5017382021692373, -0.43024138157352726, -1.007424454847146, 0.05910326956626628, -0.2013485322215839, 0.08917067745572202, -0.7205899793640858, -0.2723008765420336, -0.8741204257004942, -0.0925739340450377, 0.03690354168635003, -0.7020737238855411, 0.13118638867422838, -0.19265834076267366, -0.13458235063353408, -2.1277388895412184, -1.360066937767801, -0.8874074618432699, -1.7589304268333474, -0.8130408847660286, -1.089260285126086, -1.7918459379978393],
[0.5222055142528096, -0.07409713922970913, -0.29570632620824694, 1.5575242001695717, -0.8314104154127704, 0.35472003417494863, 0.7219925677719055, 0.9609361162822441, -0.5685523849397304, -0.798097165883804, 1.3962811433357976, -0.3783673700511255, 0.11205305688491758, 0.2756344198388654, 0.5878136862692497, 1.1664373396660914, -0.12445939910347696, 2.5581628541512975, -0.7546874537332635, 0.6306355524305169, 0.1504925359328186, -0.1284673565482903, 0.02410718608188925, -0.5436617101629985, 0.5745829925958004, -0.8921702589155714, -0.0501425292571939, -0.0886788167081173, -0.22150206021484273, 0.9529621686213956, 0.7529284335076988, 2.641721798233871, 0.46511079355976076, 0.0029668295307824157, 1.0920063972743148, -1.5798561573981342, 0.20441170473195083, -1.3879494480173968, 2.7388711950131914, -2.2862177293887136, -2.345344458544346, 0.6544664842534614],
[-0.12398019197889562, -0.09010266528267981, 0.21577433373669913, -0.13166247786877167, 0.12258945641541605, -0.16177298804075815, 0.010991209324544756, -0.3551247278822376, -0.21216534490091613, -0.014407160364292635, -0.5105508397516881, -0.1434720766647738, -0.23935797566839503, 0.10116027417899699, -0.308096964987025, 0.28458104174836035, 0.04226409296291696, -0.2645448951376646, -0.021060174995125233, -0.2587811287885342, -0.13478956390174712, -0.2570543961761899, -0.18817403885200312, -0.09588655367994724, -0.6217036227072605, -0.4782688081420672, -0.44687597900796694, -0.12286387668006395, -0.7374006558641775, -0.036074469892375824, -0.5791510588697496, -0.27463637803648566, -0.7019371648313147, -0.4967002838629952, -0.504713955440933, -1.3751106827242516, -0.9126773735230043, -1.5128938336058118, -1.907667274055976, -1.7176102699982052, -1.5419227288434285, -2.0072504194408585],
[-0.008565608699373884, -0.027199270395736393, 0.04857615633631249, -0.20292682978293872, 0.0010887232305551353, -0.042583221777513953, -0.3240188036832793, -0.17443277590120526, -0.28382115200082064, 0.1687528240188321, -0.2910983130907482, 0.024666012493927667, 0.023642387952904183, -0.32994359933522194, -0.04748715551266159, 0.20432181775463887, 0.006449336752487829, -0.1441685102737614, -0.18889546819474826, 0.2986092310953302, -0.11637175234072947, -0.6944827159328905, -0.5158804461528044, -0.0481416613005907, -0.5139756749541712, -0.5205817695574781, -0.1426804377959055, -0.4049841528006131, -0.6819746128367562, -0.0016379008862742628, -0.7603723758699049, -0.7095689120493666, -0.5430756428297544, -0.595408185274582, -0.5435634731283883, -1.3503929518995477, -0.971678386977947, -0.8586272248977298, -1.640114242483915, -1.5299186291308904, -0.9902288135176818, -1.3290151661492278],
[0.2084392351658238, -0.030754084931140624, 0.05415623361501372, 0.08120314382119782, 0.10776897525141547, 0.08681014957657078, 0.011952741847022334, -0.012984027364103415, -0.020637539766603683, 0.04099069362036618, 0.05415782986900477, 0.009840007436426811, -0.10467237309779379, -0.24099486205147122, -0.08089690093806758, -0.24684379867255823, -0.0845352325167264, -0.12903827804595758, -0.08373334499252105, -0.27975954929041974, 0.010301275839574978, -0.19743289373947884, -0.23052482964566884, -0.1540602056124641, -0.33727401164281157, -0.1352076447257404, -0.291012979103978, -0.30284701087197113, -0.6194802573657415, -0.37261686570477964, -0.4014777016622567, -0.47417352723363637, -0.5762099799173251, -0.28044966938481247, -0.7124356407675391, -2.060723181901347, -1.756154100165279, -1.963219079535383, -2.321595564194841, -2.190510979038401, -1.8109266180613346, -1.9705322778068053],
[-0.1237880281865999, -0.16185867439571594, -0.02001764653486768, -0.4616565689875395, 0.31603800859984954, -0.016517083243101714, -0.1894913914499905, -0.33217453817519693, 0.08615517836389909, -0.03480328797386694, -0.43351537441164273, -0.017598520907347502, -0.09731071956364515, -0.29537889631884356, 0.37328415354407407, -0.008979227870195006, 0.2307961152023435, -0.46378333306734304, 0.1622198973766661, -0.3312824504331648, -0.09635402108007156, 0.024950709319927832, -0.3983715510760006, -0.35719217134411224, -0.41725693080913684, -0.02811142866659872, -0.2434267442097537, -0.08995999738855943, -0.968272359430031, -0.45189271961794375, -0.1937502944965643, -0.1640932014410504, -0.6389288781088146, -0.8748231892131001, -0.5029122743990238, -0.06743662868235036, -0.8371353731501588, -0.2221395248251992, -1.7303682262034288, -1.198844483609648, -0.9853915965690018, -1.3013639094909981],
[0.03421121205764331, -0.002577992622062039, -0.11210860656111103, 0.05249830078493101, 0.10630179322238659, -0.15622777317963843, -0.027997883713174302, 0.041841131517556716, -0.1866539517588567, 0.004351558112092149, -0.0954649945662686, -0.17051899295788203, 0.07516160102330999, -0.10380159323701295, 0.010186254183506358, -0.21240886681722265, -0.21288359989315664, -0.25386715541280025, 0.0012609016369738264, -0.17476358696227673, -0.26125980964609696, -0.27649704388975044, -0.20767300804587852, -0.08685902840869936, -0.6073364818802782, -0.38914389784118625, -0.34289954851764787, -0.11490818599280922, -0.5619095900911932, -0.49641913862245035, -0.28543821813575254, -0.21114086097752494, -0.3572258746357113, -0.4625053943798244, -0.6756287337652709, -1.2702102507138227, -1.1368460808514869, -1.2264602624990806, -1.762085877027948, -1.1971506592165273, -1.049276375352573, -1.3063940740111817],
[0.20175132234187068, -1.0364636653080674, -0.011418049734434116, 0.15582118534681205, 0.8427822800311493, 0.06892739271696452, 0.8439836362254641, 0.1771757077548632, -0.5880654595347896, -0.4306713808011169, 0.15819741259580025, 0.859083124997719, 0.35564749600557927, 0.19381222638476595, -0.014425708173289323, -0.8304335737752045, 0.20810701141852073, 0.110745423946459, 0.7184178057896192, -0.2579376645336904, -0.11461856181519117, -0.12300871353489004, -0.7450808653470331, -0.09412577473204768, -0.2836342445873948, 0.21751602282694682, -0.45280920670415203, -0.47950756986698223, -0.2927244679590569, 0.0649330114543691, -0.16341644554768742, -0.163988804053327, -0.6459574700335996, -0.8495346623510021, -0.7750287647765369, -1.1060600313426494, -0.8260584270108048, -1.097487590592521, -1.5756147051902452, -1.4241422769845264, -1.5106271373748157, -2.5681740092350775],
[-0.11473683091017396, 0.02392806553533296, -0.03333046333661096, -0.25665773699481703, -0.1130083063287562, 0.3780706154383519, -0.3750948364313666, -0.16386666116406998, -0.06968801032697977, -0.03312396788962511, -0.14146832129472056, -0.0719224779942314, 0.21978826884821778, -0.1887877461176846, -0.18335623323853614, -0.08186485239911379, 0.011912061388661785, 0.020424362522815966, -0.08316577728382227, -0.1282719636754868, -0.09778007442035375, -0.01516872756647784, 0.12904646678341034, -0.05868336923040997, -0.29222575782640436, -0.11558267741392984, -0.1699711641870726, -0.02280825421777045, -0.6238315328175844, -0.2553338303120689, -0.5611082502186268, -0.08658758333248628, -0.6065054224658489, -0.5529284731392919, -0.6710225700243422, -0.8935270971621142, -1.3045386276990654, -1.5361862866188047, -2.377097703396441, -1.8526874783570444, -1.2256121680807968, -1.5366708860999645],
[0.18384370813806725, 0.015501375042588805, 0.19252238156131377, -0.5761472948397651, 0.4284657781026742, -0.15654607166351767, -0.1962773599030538, 0.04206088713386398, -0.011556219790625611, 0.24747683667206868, -0.3491877644501501, 0.12819853555577884, -0.32930046547629843, -0.3074874489456061, -0.017884534407935863, -0.29417139996782027, 0.22263833450153173, -0.48101300319633766, 0.13590982404746332, -0.35927352856652506, -0.16454568661337315, 0.036531996281064814, -0.08174963995178164, -0.5944144809766428, -0.7396390759238901, 0.2986425762613241, -0.422445320881783, -0.44102133531923926, -0.7968276678566897, -0.575753192004638, -0.5260958326780071, -0.19985742215552646, -0.057101958284334835, -0.46392828619641924, 0.03415114759625143, -1.511309540161485, -0.5625263882074887, -1.5773796762492165, -2.060593741289547, 0.1509146089513126, -0.8198711567019854, -0.7393891872990402],
[0.1528544111037656, -0.04279069342365847, -0.11844627631427848, -0.41268503891516195, -0.16690976902780807, 0.011231721649135528, 0.020022887503391495, -0.06043166391723888, -0.12521624141405974, -0.10608111360545273, -0.20524225163651733, -0.20196354042411455, 0.09806211660079542, -0.12261745511294607, -0.042520472820289125, -0.4800834858256395, 0.18156452886671895, -0.5318754370652954, -0.20782317554800728, -0.2801460436421006, -0.2167658827067864, -0.3262892940988947, -0.3928120261041387, -0.19260908917806502, -0.6651298009314736, 0.020521654581210816, -0.2615057543626333, -0.2552510253907828, -0.7778587075923976, -0.560313498911112, -0.5924428596832088, -0.25103336952666466, -0.4831861800385417, -0.6603119389856991, -0.768248985041996, -2.3986188185707027, -2.116930743875921, -1.6598401588806113, -2.894079353761489, -2.0133666173368665, -2.006936259750524, -2.3464820877231145],
[0.17009650450859842, -0.10422292190125372, 0.08284734458583431, -0.21371551902623434, -0.1132482636282335, -0.20641581864697503, 0.30367538999951155, -0.0007504513638220067, -0.32267325818737175, -0.017044970741280044, -0.5295539893420738, 0.31294630905871157, -0.13394116711178625, -0.19791526486970898, 0.12215073464678522, -0.32832295816966367, 0.1533953416594709, -0.5447633453703692, 0.19257052881700062, -0.5390225696244046, 0.09630111434045613, -0.001611216669039855, -0.4196163842013964, -0.2647073980270639, -0.4905707268254313, 0.3168742779501993, -0.35429399736188233, -0.17340698583584696, -0.6979039408333445, -0.8003995662627345, -0.23716755491668307, -0.26966996826224127, -0.5045742704264182, -0.7262487802984885, -0.5505536934441684, -1.7421483789793457, -0.9477287248826006, -1.590129169262807, -2.009902562530003, -1.2348228868526796, -1.5606267610338507, -1.97037242820555],
[-0.4516111434162585, -0.021844964414661633, -0.4178387234702717, -4.8551428798453176e-05, 0.022876388433543997, 0.29337482952157523, 0.22081662485101045, -0.1616622078796856, -0.022463490696912954, -0.5047279102237843, 0.10772289484953196, -0.02149189177088008, 0.04068319948374202, 0.41144461579159525, 0.08854108435808447, 0.2821546695770571, -0.10932292519301816, -0.08867989369598377, -0.01876384451901837, -0.1829531152625271, -0.2274766576521198, -0.07691540838398211, -0.49500555670230273, -0.13803496757806466, -0.4124845124217273, -0.31522930761756374, -0.43091703689637256, -0.17718247727039227, 0.18883427849331155, -0.42148381613902897, 0.2522466320183296, -0.5384852879320406, -0.5281673037641513, -0.9706150516994149, -0.7810398511568912, 0.940965936110365, -0.8413752668603536, -0.2602855956003419, -2.0696183141644244, -1.292909148199027, -1.3509006306423748, -1.4121414989269534],
[0.29756793454354, -0.44141143536893424, -0.12489475851643989, 0.43703454287682086, 0.18928462845013175, 0.6650994600709649, -0.12048811508148703, 0.16010382617372587, -0.18840197367711778, -0.47651516601267996, -0.05601247282905125, 0.3716999826457987, 0.02142971114496921, -0.20645838167096742, 0.18571181277785362, -0.7272236451558655, -0.6925706259398543, 0.045646458192914297, 0.01450897019651745, -0.2166920103617781, -0.036024189690194344, -0.511241382214639, -0.7372513299782216, -0.2148666035494555, -0.1320400602917843, -0.6007833684957068, -0.6156917161729308, 0.03421605723667298, -0.482649939610956, -0.6747200081735967, -0.48206292971651254, -0.13104359956753597, -0.6652735174852293, -0.7382682698984421, -0.8427251156292305, -2.6429258592987375, -2.2589654893034408, -2.3607572707457156, -2.84053117165624, -2.5254374535098827, -2.389536729695756, -2.5113238633651114],
[-0.01112127503815587, -0.20352153894404063, -0.07152171414735457, -0.039328184248280974, -0.2019967168705688, -0.23394706052623612, -0.16684884726403668, -0.11414177221700744, -0.9705784574150373, -0.2070426231676081, -0.15265383310523545, -0.13473215717120823, -0.18616031289576768, -0.28426118807673006, 0.10661259941936387, 1.0771955474466512, -0.2740688997414359, -0.37266404066262104, -0.2911718036330766, -0.22347558224386901, -0.06688438675184868, -0.39400020049309503, 0.30980038970898993, -0.4197522010947602, -1.088761887058689, -0.48980820988517604, -0.4947636370019016, -0.2560571295799335, -0.7040580425019652, -0.24452079365149249, -0.7572344944690349, 0.30518756337064207, -0.742440463306243, -0.6861562527072547, -0.5636514191712313, -1.1483873215350007, -0.5762478513947407, -0.9842705497512787, -0.6917047987950115, -1.7220805422404275, -1.7639705927266982, -1.8749843332032616],
[0.03144647929518221, -0.06834472930461004, -0.025213387987018353, -0.08085725996566911, -0.02993776450500165, -0.11936163139830207, -0.026448298639266794, -0.08920778747754789, 0.07427692181540853, -0.15317101959080853, -0.1082206525540505, -0.13708138847819534, -0.12163297800605341, -0.1343232071016975, -0.20722698477992568, -0.47494328161276195, 0.02159238256428505, -0.45474106430778444, -0.26583493952867454, -0.5517754204567072, -0.1810747921573276, -0.12392758528452229, 0.09812521345240638, -0.23451804766627637, -0.689233517147709, -0.010214549174657462, 0.12691065881332808, -0.16526380592141185, -0.47454712167033974, -0.6430185453468399, -0.3567481759769854, 0.004784301376602445, -0.5697238585546424, -0.588418289974781, -0.3071004375671718, -0.8315228640941225, -0.532046982460897, -0.3347344588552225, -1.929041001177848, -0.6619026034029649, -0.21723819687680687, -1.1777844976973328],
[-0.31037393750396103, 1.0631263706135285, 2.1129520815062404, 0.3246799771712618, 1.1119029882946156, 1.1917864991881018, -1.5351976091338477, -0.4448862747670516, 0.3457040557320539, 0.575601893783132, 0.48491262979617533, 0.2847307399334848, 0.6157004216155743, -0.7669332601649088, -0.34544389233004896, 0.42984012692306045, 0.3402097023888919, -0.018805628559763757, 0.2011440733302975, 0.4443434388985859, -0.3406642211333884, -0.19447387812280376, 0.16019671563613364, 0.0072428225736924385, 0.19930957901795843, -0.4689196541106822, -0.520713156568054, 0.12912691147175218, -0.6164864852806304, 0.2842896355150656, -0.9440894707278605, -0.606740475340588, -1.1163642345211942, -0.9769285669150772, -0.6589791572840773, -2.8655261504995595, -2.5363739929418885, -3.2647894497458188, -3.3196477508162485, -3.240652939688674, -2.858741778125113, -3.135056261366421],
[-0.05053508248514265, -0.01419786293084266, -0.14568372947304625, 0.01901623338140011, -0.2045744793204934, 0.0949492032476894, -0.06505271093867034, -0.0503483780922286, -0.05891715179599649, -0.18265098684565917, 0.0327106908739856, -0.0855525442722646, 0.08377613541324248, -0.1475926576772878, -0.3038527897694679, -0.2584202964854224, -0.2117877650790714, -0.21124066166524605, -0.17738295267938017, -0.2212322708609183, -0.22717515250111087, -0.2754089438732761, -0.3778202656962322, -0.15661369366463593, -0.3485468657602185, -0.35896729365619656, -0.11552282842916341, -0.047823304136677104, -0.6691056278008786, -0.49777082783581755, -0.4012461757591947, -0.232109707975473, -0.285564670413969, -0.5705015120834932, -0.5009773670370627, -1.5798445383792092, -1.3045950798095702, -1.3705163007166559, -2.024010217135923, -1.3823134965184414, -1.1481545686614294, -1.4383762327830294],
[0.1825931085597441, -0.3315155420123455, 0.03265011378682041, -0.13495812978891716, 0.11406249105749143, -0.2156494534632547, -0.0713489427722153, 0.08402822615785774, -0.17027555073244782, -0.0978218692102602, -0.2846712955068164, -0.15050868812252596, -0.2648814688815325, -0.2151323538030211, -0.007463842852541993, -0.46299918769706744, -0.27620646216255823, -0.4163666081625524, 0.3240527060811471, -0.4418942774452318, -0.10069134833008636, -0.16595380942553262, -0.3394682895211578, 0.11407003184669921, -0.3735183829358835, 0.03451382474455683, -0.35420847810564793, -0.3849137775215737, -0.3676103566120294, -0.6267577975679539, -0.5812275933623621, -0.42151194813508014, -0.4011879088011997, -0.4987156529090158, -0.598124966769446, -1.9732571423208018, -1.52040730246756, -1.8603169113640212, -2.035591823912722, -1.7000825460308877, -1.5347839795213372, -1.954965305984288],
[-1.527606906396132, -0.39041011235571477, -0.38723000620974546, 3.9801335868572885, 0.6014348393921989, 1.2507152930580052, 0.046708905765787616, -0.4654684517559853, -0.6605340952920385, 0.21801272421224888, 1.2505931487670825, 0.1701765895805803, 0.521492145523336, 0.09894678439040396, -1.5685340532712344, -0.9013149520078924, 0.2695094331547238, -0.6308601232567989, 0.12656618094415512, 0.0741515478791662, 0.5336602063419377, 0.7120290268998815, -0.3949159218015413, -0.1742344665573241, -0.5840233827060634, 0.03529967515287228, 0.5892712423632631, -0.9246721136020175, 0.9645788594223674, 0.23830991567888232, -0.14536798523480662, -1.2910161813732233, -0.5798246887123968, -0.9955978156772135, -0.32338918361006364, -1.873507843222537, -1.8362104448724978, -2.330028612664706, -3.1086122316538236, -2.708851169350101, -2.430299266389264, -2.7799652019044307],
[1.3109768506115635, 1.3366133818876578, 0.736702979504321, 1.8457601935168038, -3.8564620201581707, 2.378064042215934, 1.4278698898891602, 2.0145758045393127, 0.028891617739506367, 0.6461032259734298, 1.4016236398233328, -2.071521203809601, 0.6513530100902666, -0.09433633853105863, 1.720943350078073, 0.12018378329441569, 0.9760353108891849, -0.6867445565107881, -1.4288504049547572, 1.0112534233083976, 2.4526128173711386, 0.7018584630173474, -0.6504866574312788, 0.3671691050095679, -0.02548002013586239, -1.6702295348002618, -0.7118044848459567, 1.2937673440367052, -0.12064576088445349, -1.1849325428408002, 0.05115507924707429, -1.5366620332298073, 1.6418052559956808, -0.9146686722637658, 1.0560706578338515, -1.5777095499380338, -2.438832717949051, -1.2641289890822252, -2.466203475097924, -0.970531631690902, -3.1925327321435057, -0.30784603528678683],
[-3.6558401082982126, 11.121672104056495, -1.8485878619915879, -0.3650721325250142, 5.176132032534697, 2.636668574517547, -3.7824696173464343, -0.6459036957760879, 3.3672771538312167, -0.8153158686221698, 2.1850762307223777, 1.975971271783044, 0.7391834358955235, 0.3501946270683219, -2.393825252243056, -0.01867411474173345, 0.5849899508812862, -0.990744317896938, 0.16907093573493617, 0.9196986275357146, -1.2799447132339683, 0.8557969858638064, -0.7934740215565408, 1.8615048506150378, 0.18746523547219487, -1.9321827743629851, -1.0733914816605372, 1.75040850025125, 1.2622941703213533, -4.340327627185131, 0.5682238353961709, -3.0849226026377057, -2.3401012295458465, -2.28800697146899, 0.2340104270191298, 1.6015807075212027, -5.144426578123614, -0.5013789251063825, -1.4836847094784544, -1.8202986375560253, -1.748866735814522, -2.287514591903213],
[-0.3731904571163489, -3.403785218920649, 4.80792544926141, 0.7145881465937363, -1.5584628530022033, -1.4082395716877338, 0.09199328258304666, 0.2257983427809623, -2.441264652525846, 1.458616160544523, 1.6741535633734181, 0.6505647070727919, -0.38619813780481055, 0.268975530582577, 0.7116411025081123, 0.3012859804922275, -1.6272679272419628, -0.07694176735184743, -0.5126544817508502, -1.0979385247917697, -0.17357213008026234, -0.07130517468977941, 2.638269597059723, -2.267165551989435, -0.0823740797568893, -0.7624037013808833, -0.5374082649326007, -1.0476311641645664, -1.6345783035284671, 1.705252306421002, -3.5658680179378743, 1.5821489598596299, 0.05015355558412718, 0.7076631561568285, -0.8397909162261276, -2.672741185498761, -1.1513527757520323, -5.585620569421987, -3.7366267162032565, -2.1296286621224767, -2.7640178447125616, -1.8358044979364747],
[0.4212830735712897, -0.14881158543090672, 0.5168742786809842, -0.45682818125965774, 0.13576995484032245, 0.365756321876287, -0.5795005456641562, 0.13130013081778877, -0.5790819636359623, 0.0734384903634512, -0.6513359977771822, -0.15491293400497644, 0.7658134459287275, -0.6825022866720929, 0.22696090992557488, -0.36895012618531514, -0.2575570721311903, -0.7436944108503064, -0.6105353969015485, 0.2223745725885351, -0.10822994225965839, 0.10938747334012723, -0.06912940785276835, 0.14377982417998694, -0.35769696808722096, -0.7153624359715067, 0.21520854631945827, 0.21076192337924415, -0.9043372153476453, -0.3914761520392313, -0.4593973592899839, -1.0397977555601559, -1.041926973219723, -0.26990942223018166, -0.4076358487194351, -2.5555957963766014, -2.244616422646113, -2.4709421119792947, -2.650643419726177, -2.53942103853656, -2.4736536708823924, -2.4797985685075643],
[-0.047723113508650515, 0.07241777624404437, -0.14561863718390328, -0.3380020565917887, -0.05767190739502066, 0.14292320735275044, -0.024523158020438595, -0.186314924700179, -0.08945962263172137, -0.00719010384047605, -0.024902004432975383, -0.19048001388888264, 0.19319721971754558, -0.20082185168935035, -0.22668510516047743, -0.38182131955901866, -0.37401861366979106, -0.12133283302363679, -0.34389823566877215, -0.11316388796825932, -0.16057259129156273, 0.018230252146257914, -0.4453472804093933, -0.5902025243306521, -0.5974858716196189, -0.7355380484860323, -0.2523744511959463, -0.004476559040712642, -0.47950069659344174, -0.4514017774743329, -0.4631304193684104, -0.28250944584139176, -0.2504173532977062, -0.43948442917332015, -0.4866353430926342, -1.4747187873828274, -1.3300023576846272, -1.3179024611271415, -1.3862287822585047, -1.1175589283375569, -1.0747990149553956, -0.9918481546739063],
[0.6045250172047014, 0.4117878202616513, -0.8540019961286859, -0.6223839713837883, -0.15879381584894345, 0.13123179396599294, -0.06730924808144377, 0.27190771257982643, -0.06915565705047108, -0.6626796176593841, -0.3925994663301831, -0.259400653942046, 0.31768373823415014, -0.30129130390784287, -0.13964466519661126, -0.17223566920833402, -0.19611177348372985, -0.5157027324950272, -0.08571166502006476, 0.04461368582286601, -0.3457000590984071, -0.31460700220127874, -0.4052313309657127, -0.6688207579162869, -0.07979415711347568, -0.19798851104133472, 0.32053044664885877, -0.25205542583101076, -0.7648758672931519, -0.9516501010396036, -0.2007836686923709, -0.18216918370909715, -0.3538589446098576, -0.5290319124536605, -0.71501482605826, -2.2695670135785666, -2.0255389111787157, -2.1261361972713386, -2.28550220212355, -2.277273186036577, -1.830599633710499, -1.9517456535437172],
[0.5561118798043226, 0.8562202102520102, -0.8226885467351612, -1.7903980995077218, -0.1264425789136824, 0.2315900930461192, -1.318037699165097, -0.9345932883114173, -0.6374382177861827, 0.3534919801865069, -1.6461921544640874, -0.6058983601713923, 0.5628355947622791, -3.983308817241939, 0.2778537680008432, -2.4701411861401255, -1.7772201061888881, -0.25853140920480855, -1.5670532413199922, 1.566888347175815, -2.1137744675880747, -1.481041719322257, -0.3259352362878626, -0.3764505931690228, 0.07050917467652025, -2.010776983356286, 3.969963589337386, 1.480642279908515, -0.8372370514975734, 0.9934602807679156, -0.5896849188341203, 0.7632237004784928, -3.0553779823347242, 2.8363581410403875, -1.3907535226248773, -1.1305122170843664, -2.112875434695618, -3.057011487619992, -2.1478473862798673, -3.6230838017935194, -0.9337486130299717, -0.4778427597977103],
[-0.29256117950437105, -0.5810181023849507, 0.864887954538538, -0.18084433342139097, -0.08697688328721002, 0.3834733641562775, 0.343631371596226, -0.5425798445931574, -0.5634366803944919, -0.42603392454265665, 1.9061525056324984, -0.5317696029012735, -0.08945907278944783, 0.5264249543969532, -0.25148214625157506, -0.4356587950123555, 0.23050003248369488, -0.006570343780669767, -0.04305827638603564, -0.3422719102653929, 0.24698915303364682, -0.6984123557929038, 1.388756230724511, -0.04961775301712691, 0.24325457204532566, -0.716797379339168, -0.5791331366133404, 0.02275891329737502, -0.21197291189243625, 2.38782638897633, -0.8699664943902495, -1.63678094982754, -0.10319711163722659, -1.099753878918518, -0.5988788773077978, -1.5105707026338804, 0.8287420741356452, -2.2450979965277456, -2.919423547983808, -3.024480719284279, -1.8222867629900399, -1.0837514330386067],
[1.190684990236935, 0.8968114731014096, -0.39759017491050663, 0.8176149019133476, -0.5477310806890807, 0.2708089369009753, 0.5729835694604613, 0.6398047937565697, 0.4330302453804394, -0.5569993724845669, 0.5096736049040858, -0.552828698691963, -0.22247161356521927, 0.5064809588748969, 0.21153587886936998, 0.0034832431452271355, -0.10852210383952972, 0.6048444010006827, -0.4819166109248479, -0.44137627660953543, -0.041573325014977985, -0.3704913845329974, -0.46360830470425096, -0.5079867248058872, -0.7290188948912684, -0.16077849541240413, -0.44090683223702887, 0.08341794964968578, -0.6652393339010751, -1.0366175495443273, -0.34252479155654686, -0.2673166727290777, 0.6482587437280036, -0.5615531594335809, -0.3768051207070117, -1.8722493316074063, -1.6100164356589317, -1.8030734964210682, -0.6492769477425483, -1.3030976473534719, -1.4697293366684967, -0.559375025385892],
[-0.031334009307891014, -0.08422394020558471, -0.02861352816177834, 0.006823266084860057, 0.02486509349122966, -0.11387379183545113, 0.025405684974626298, -0.03774496562030545, -0.17754210276606183, 0.022703253511149272, -0.2591156371489086, 0.02117874705747783, -0.16286167884471783, -0.09130558655804603, -0.16403353204278406, -0.3561724062946038, 0.1665210612010843, 0.006423954152860457, -0.3718023960428928, -0.3822588116259174, -0.23767980732554866, -0.19820702232844511, -0.18261820675305307, 0.10875186269451777, -0.17238383706325183, -0.15933825187043035, 0.004980352513003093, 0.046842996465745135, -0.5211639451815027, -0.015743688681577998, -0.11630738674043338, -0.340567562132799, -0.32813735015058887, 0.06982568246134735, -0.0981450924024571, -0.27763284766655116, -0.05974971620981051, 0.19227572998895315, -0.9506684033969005, -0.5362305571418439, 0.1290640075493577, -0.31141852600788894],
[0.21174875547895083, 0.017602807720730584, -0.17986869290784627, -0.21052563446067418, -0.258015855852489, -0.06575574209417834, -0.1392973776435863, 0.13058023745795588, -0.07099549292231308, 0.02771366699721861, -0.1191638329387721, -0.3974118990265639, -0.17527272895745774, -0.2726370422316531, -0.03998853421835949, -0.22891315289730804, -0.15860144304873836, -0.11332043122955608, -0.2621006405343554, -0.38242112258669275, -0.42367940947273036, -0.2439642360949186, -0.34251081925101506, -0.3708128213032933, -0.3583507164798782, -0.6513219751271807, -0.11870171685573107, -0.05688203584557555, -0.8329675375653612, -0.6717253741093143, -0.4316417104006481, -0.15357052410359515, -0.5234727134540519, -0.2937625737711696, -0.45049613008471145, -1.7798995519602192, -1.5441437531652142, -1.75702450669325, -1.8722287782690323, -1.9330916785294832, -1.4756657047560668, -1.6376558925069393],
[-0.0890442818991385, 0.12272794752169396, -0.1935027521558569, -0.15083640568743506, -0.17553538506271815, -0.009130611903613447, -0.1186653350749734, -0.2349832887879201, 0.04587966915990948, 0.028591538848643135, -0.17066356872919689, -0.0035589529296828843, -0.045636112734793836, -0.11895491643719265, -0.3215511838356355, -0.2758798796044619, -0.32644523343513954, -0.3198109073810296, -0.2407488396566647, -0.265818157237954, -0.27325385817046044, -0.2023127129982758, -0.41371545360558126, -0.42711652495344943, -0.40376207637202705, -0.4710825644243404, -0.3717709866433864, -0.09883642064929887, -0.37209539054984536, -0.39613815023081245, -0.3879191361054959, -0.20313356290603604, -0.33369933653633943, -0.4246306960757423, -0.457374579173498, -1.1141912217228243, -1.0774725196889632, -0.674029434199846, -1.2405370038520593, -0.7998286465874634, -0.7816682320835397, -0.9900697595736904],
[-0.09311921309509215, -0.2629215202342256, -0.1370724111282271, -0.3409136678896097, 0.31921434573550145, 0.17330354278082472, 0.022733869074230275, 0.016915192730493706, -0.29698884421975524, -0.22999526460860414, -0.2540401537158546, 0.13103471163317518, 0.12501798198121147, -0.24196142051035452, -0.13184973837572367, -0.25978336819810927, -0.3423791095013798, -0.3735480648039333, -0.03484869484272121, -0.028280350442078325, -0.1726706441167837, -0.2866508162173614, -0.5049080941911915, -0.5112669489320512, -0.5547096047971921, 0.08506111769406216, -0.27610413592688593, -0.20394143436325335, -0.021235489026902032, -0.5498930039674701, -0.21922840207410657, -0.2365532685850706, 0.01158597700791373, -0.43219495441205014, 0.20043721186301594, -0.6082747696632339, -0.11022797965391884, -0.6838698653195872, -1.1217705891701213, 0.03225128164452439, -0.4067739072665635, -0.3560292494310238],
[0.6798964937944776, -0.002233627233861687, -0.3007324115763478, 0.14908580318497683, -0.9084577165986091, 0.34119323560494147, -0.46345418313105946, -0.44217467148099954, -0.6824140234655179, -0.5441425239496628, 1.624950982663372, 0.2373122051373233, 1.7227419292476218, -4.1895250683311245, -0.13428015316740027, -0.8666331962187162, -0.9003397795349264, 1.1197531687820623, -2.1467629546699944, 1.875703122945023, 1.2544423444783557, -1.9551476587917072, -1.4228665479404436, -1.134704843360047, 2.84060708796354, 0.8041546161765175, 1.1747506535089676, -1.444182090065313, -0.23267403433057712, -1.3626425626059515, -1.5129872789840255, -2.217332381167728, -1.97246422733763, -2.4639481229921443, -2.187730555752499, -1.9755982029846266, -3.995459125774435, -4.578952802087102, -2.263281442210975, 0.03922779097502346, -4.01151659500433, 1.0247985266517021],
[0.4124876614362255, -1.144466767422251, -0.6230029468369158, -1.315070672139193, -0.47249746153693223, 0.2059181269908579, 0.3556464349788612, 0.2221703289904552, -1.3151686667766473, -0.6918596381871687, -3.9834429994164764, -1.1156363277167642, 0.2144716461538189, -0.27141089068554647, -0.6433630875677012, -1.590512967850867, 2.5933208317222887, 1.6677939798591828, -1.973992714386354, -0.3934671835755736, -0.5747525876890388, -0.48599348569552103, -2.6548771817519246, 0.18536984253650712, -0.14697859027830915, -0.4091824696272054, 0.27239039897711387, -1.831275393813116, -1.1247584444385579, -2.0103611194046063, 3.366837537859616, -0.4334938655830044, 1.4281711609787526, -1.892622370636005, 0.2350628363988976, -0.9433574331314245, 0.6357448211598578, -1.7406798231135425, -1.9939501370845998, -3.483786848531534, -2.642656068289048, -1.1195822616118067],
[-0.15974292012711622, 0.007098340011791697, -0.1293361791160403, -0.32106191755896674, 0.028051262446760147, -0.11172283519804894, 0.02538717426019432, -0.09357448568162109, -0.1355670682817703, -0.2405081472436626, -0.5847097924507726, 0.013448271262784908, -0.24043480910802795, 0.0985567796646826, -0.16650671555441665, -0.189772999306855, 0.040829322915790146, -0.45005860106203865, -0.19416712423191118, -0.33903202905385293, -0.42785582035603836, -0.20292999279950832, -0.21745287935348437, -0.42052462385664313, -0.5905221673724579, 0.0005017592609291015, -0.48496742092407075, -0.10672371427962545, -0.7663436188887579, -0.7791898554760016, -0.42514993430829806, -0.022635493613941987, -0.4033485448910758, -0.6496700561840313, -0.3747899890487753, -1.9685158354495076, -1.3977998568260868, -1.7148957421635302, -1.9786092409179898, -1.5198328558554262, -1.40948278854579, -1.748377533980267],
[-0.36665148163886235, -0.0316324471697894, 0.14763884522647086, -0.1669131843524509, -0.04222099285239897, -0.08044925739682401, -0.04204787341497238, -0.2364919006793915, 0.01275987008652857, 0.15011677209291446, 0.06276969136504844, -0.03072153002022568, 0.04309666692311944, -0.21799218225117445, -0.2412936156108071, 0.13659602604196672, 0.24344954201834876, 0.1300296206886888, 0.032250749410441204, -0.01115779663106605, -0.13355948699683995, -0.2140863434001557, -0.11630171343138739, -0.09663325757113002, -0.0984363057874446, -0.42380478390750514, -0.32441756021489304, -0.12539817930684855, -0.8566129046835075, -0.42991785548235134, -0.5413937015798599, -0.15194313584374203, -0.5211666654459849, -0.46997410840066156, -0.9419309931272393, -3.2265712477723016, -2.914682803565263, -3.3330602058910013, -3.526827414290847, -3.305652562176362, -3.0324885654297615, -3.2140757463338674],
[0.12462663136249305, 0.12430763156669825, 0.08282858676556878, 0.10308673424988422, 0.140870385967903, -0.13503081187793023, -0.12618882921844965, -0.010707867580726933, -0.11964694819413148, -0.21277635636686254, -0.12395973132276138, -0.1475271558805604, -0.3744619025279689, 0.03807781500959734, -0.1328243372729125, -0.14198072969798417, -0.3515083173467349, -0.02503377625326441, -0.3585714916734906, -0.2571179669454799, 0.041306473056881815, -0.33946416532904267, -0.2645871876431484, -0.21242826308501817, -0.3494990204559044, -0.47457039595700634, -0.372162486021284, -0.3453321297797882, -0.7432934429850308, -0.5091329541694153, -0.6235518327280891, -0.156133374365804, -0.44134875475368485, -0.5950585904635037, -0.5963320742225956, -2.0888028011110613, -1.8923217136905062, -2.0352188616386133, -2.5107170201450595, -2.043432948768078, -1.774627333656393, -2.073085351115507],
[-0.048542379536841936, -0.34291679811205783, -0.025041272323646706, -0.27792651323316747, 0.011082482627695489, -0.06292737577446807, -0.029284257347773946, -0.11237789165765198, -0.11084809854367576, -0.019664711664221766, -0.14421262152225642, -0.04740361039788097, 0.05113879132412878, 0.0441682015715918, 0.0623409063680223, -0.20811277490801247, 0.019254391213243394, -0.30844267439461326, -0.35757105082149143, -0.3020200256575922, -0.256579061543816, -0.16480196770773167, -0.2713546817372426, -0.01844092859551007, -0.2243209002471551, -0.5150115339913558, -0.4447580782437222, -0.16595594080054463, -0.7002101579474135, -0.6273017071534298, -0.3944937743520489, -0.20716286718348356, -0.5319918440603133, -0.6580742575881847, -0.5168769950995946, -1.8948003820223944, -1.6033527588936614, -1.897400635474442, -2.1755821640244433, -1.9479827195230632, -1.539770057924445, -2.167061344903691],
[-0.12031385248501125, 0.02792359999473707, -0.24793197601937117, -0.004078952574846141, -0.26609642660537386, -0.022585895095143683, -0.016289079274892513, -0.17792724753799602, -0.2121293825031089, -0.009813360117278985, -0.21169103328196365, -0.2249240922931987, -0.28965784259389593, 0.0041089651507809286, -0.22322130658576433, -0.08231627264330352, -0.1424333705652053, -0.5302475402615267, -0.20340764130490036, -0.47183341376635535, -0.3177442236815622, -0.3056656354954733, -0.15785756755161898, -0.4715320538548597, -0.28331612798638434, -0.5024699460504511, -0.5377291803873426, -0.06798828488264347, -0.20555268562250578, -0.6875760655020436, -0.3282266376445247, -0.020575940945205626, -0.44940378584515805, -0.8307182494010449, -0.4619634644505696, -1.0415430243137287, -0.9392583991600036, -1.159752942237566, -1.6690789221426514, -1.2424050811760692, -0.9113951908133772, -1.3396399836767905],
[-0.1517117952019397, -0.08876670538560572, 0.012150103340415839, -0.25154753482278125, 0.12305340796342948, 0.003756674316768769, -0.13621080548718018, -0.09915890779480567, -0.20768748164954468, -0.2534665193730988, -0.2009735817895099, -0.1326573708981096, 0.07897224150586554, -0.03264809620284406, -0.1516990752533594, -0.26941740257808394, -0.05120995952836276, -0.37035180317587757, -0.24674422758409642, -0.48917157284855534, -0.2295456109849622, -0.31382132020157427, -0.1380456202545731, -0.1545104123153531, -0.4803266300951863, -0.23600332709246077, 0.20559649068238092, -0.0932287453333365, -0.6375165947718465, -0.3279911695294224, -0.45924873950181205, -0.3077280045061227, -0.5818848210237729, -0.3279245515773147, -0.7152200054304776, -1.0399924705457588, -1.0596702937066846, -0.569248182397246, -2.2463925294577463, -1.3180877664923476, -0.7506392487575982, -1.3717087553827023],
[-0.004221440566216553, -0.04304850618617948, 0.12572939519853477, 0.04402133126790379, -0.006984747812437168, -0.046527709000303265, -0.06388774170301702, -0.08597457444256187, -0.1718556872666245, -0.017893131588963537, -0.14831993686556508, 0.037727953937117385, -0.06715872036851166, 0.011763636057291117, -0.19922360901929767, -0.24308978875903262, -0.02095525602313516, -0.2936558692890608, -0.06263019803760111, -0.21956452389303888, -0.23602013552383863, -0.3398308087050425, -0.10138231533412305, -0.22020044909497374, -0.3562492590708695, -0.1782711104881712, -0.4115854407214311, -0.13895496409353683, -0.6061367667834207, -0.21255153666203508, -0.5642904710636049, -0.38188905111134036, -0.2750293817006718, -0.42165092437013796, -0.12754673984153037, -1.2834931556886633, -0.6991842809004231, -1.236162763570268, -2.0138770322848045, -0.742511166635311, -1.1159794649289734, -1.066796246960134],
[-0.005261252448566998, -0.42519718035214676, 0.06094743936208573, -0.24182530740332178, 0.3672750141842767, 0.11275607593647173, 0.06721063058990775, -0.2069085470756323, -0.33931993611292205, 0.14758986378014893, -0.4370151325359101, 0.16373835980020568, -0.14121150233473054, 0.05316127932402193, 0.04966512256031663, -0.403158128651399, -0.11547300652109094, -0.3266662985813667, -0.15804666718658134, -0.1913752562886297, -0.1076320343347427, -0.30443102450105225, -0.020551852170123622, -0.18606886715803164, -0.5003872430741896, -0.1764382268253867, -0.3469820888244408, -0.26759890734362857, -0.540653680974199, -0.03675188278425622, -0.4262283336957074, -0.4813861518717875, -0.39046573899537945, -0.27846409985843695, -0.6910807156292309, -0.9745885926130592, -0.6536679231788667, -1.0044561409682298, -2.008107873763588, -1.1167720732205058, -0.7479341824826055, -1.308802110216948],
[0.3817649348302674, 0.12117761723446323, 0.02066569994395979, 0.35589839155980013, -0.3599479367511083, 0.020126261525055776, 0.041054273196772795, 0.335695675262668, -0.1462528922945298, -0.24523015790448224, -0.13639686690893058, -0.08191549300955621, -0.23920107340427366, 0.01319974486622617, 0.05296774267045481, -0.13917350598523864, -0.2197510707162671, 0.21448306500019557, -0.39847449890644154, -0.035092610100043796, -0.39945800421588784, -0.17800871017426245, -0.5293839582152008, -0.1865696763615842, 0.0036041034056549385, -0.46317025306942694, -0.3193371623930532, -0.1649605668039669, -0.7722766950712688, -0.5967259382829612, -0.2524649622558386, -0.6230307741591107, -0.13319231833165868, -0.46197244251014, -0.648261335599325, -2.0377546037008307, -1.5183169565757884, -1.8499635567349908, -1.1436068985221552, -1.877346568206476, -1.693177529699047, -0.7681743806943804],
[0.21752479222226567, 0.18947451092000123, 0.13473734208266358, 0.034714447104576995, -0.1132386904422314, -0.2348927918404904, -0.0016771003498017645, -0.1985335102246962, -0.32625422052062825, 0.14391387773845934, 0.06509086050709133, -0.1333299149498883, -0.6424327332863728, 0.22186711373113088, -0.3929096520496408, -0.19189755849235351, 0.0009889719434971508, -0.05319106554914018, -0.03597725238912957, -0.2271183061859901, 0.15810038086942665, -0.5127893341842957, -0.6268320159099535, -0.17537273843374449, -0.458123216096959, -0.2989409588842297, -0.5260079668625062, -0.25799285240500874, -0.784616714216146, -0.6332964729050543, -0.45390714344746746, -0.24795863160370005, -0.3890546395888659, -0.017350038202862555, -0.36722529610658866, -2.2472931823364375, -1.530472185444158, -1.6092400028903315, -1.9928780352559639, -1.4249022914748783, -1.1232151441837088, -1.6943975959536428],
[-0.22734624349406043, -0.14892937643074106, -0.13127746951548425, -0.020289299008439104, 0.13645607918810365, 0.1449554230382, 0.19047614939581844, -0.2716610132872714, -0.23260245136817928, -0.2973005949104195, -0.16143133223154615, -0.14448075505132918, 0.0907826700816539, 0.03032819170042864, -0.2840731237396651, -0.14497542753597734, -0.0022776473909254195, -0.10948539543106985, -0.2083419970856859, -0.2858968575857427, 0.01865631857250285, -0.16065942916986595, 0.12211009901452588, -0.27587204072499283, -0.13536399603561192, -0.28465961763527864, -0.27567603125974777, -0.2391536143402236, -0.6979494834116402, 0.02608806745003632, -0.5128303571138035, -0.34256248099451514, -0.5250188836488047, -0.4950960471367579, -0.755711823473937, -1.5551876551376425, -1.5898068765287034, -1.994816813473923, -2.4522531381200987, -2.2394706290586246, -1.8211102114708324, -2.140679596172768],
[1.2532515771861323, 2.3594950455376345, 0.2087536986352691, -5.241537979149789, -0.020027871110594588, 0.7414603385286448, 1.7098491463472623, -0.3290351964714401, 0.4667469068666911, 1.8053982860932625, -1.7787792847603776, -1.287545818695623, 1.0454260304064777, -0.3713113919699225, -1.3931973312998798, -0.6299796043496824, 0.219203930425553, -4.916822122424185, -1.1094212323299375, 4.994396218375029, -2.167353083232213, -0.6762699498488313, -1.0096599143151916, 0.6343182534226185, -0.005128726038114811, -1.7125644538649971, -1.7731525061219668, 2.1194332549733255, -0.8621904224989722, -1.7210178997162424, 1.7420460048697548, 2.025899726880549, -1.7580412815247344, 0.7035270309173293, 0.9162143945972693, -2.3924725663891175, -3.0748879744878783, -0.07373247208482127, -2.5015336222851237, -2.705391703261283, -3.6996781145749256, 1.408936232093259],
[0.058860339440916745, -0.1076377653557524, -0.14295382315710212, -0.37641981282946535, -0.06891630577159714, -0.06009023722412798, -0.23378115889894338, -0.09908710809699352, -0.2966424065411783, -0.2038694970814494, -0.362996018767265, -0.18838238804605323, -0.0440175530340475, -0.36177440088715695, -0.0470276474490146, -0.3240983758784872, 0.3308299616799693, -0.4787223739158542, -0.11846664172314886, -0.3853943548266603, 0.3658944052764844, -0.17807630906512423, -0.22985032927301693, 0.10035906103696739, -0.4035153042815941, 0.4916039715392704, -0.15716561464312837, -0.7394441857916628, -0.5394645818753232, -0.7467942611043366, -0.4059420996199608, -0.6759174392327731, -0.2101475592252311, -0.3319153207895752, -0.12071531614683761, -0.9473005218507287, -0.3273437643306178, -1.5922399526860305, -1.8828419204517837, 0.6028706840281175, -1.4069947842366286, -1.3921873971356125],
[-0.019727819378433206, -0.5938424210812312, 0.09807301567620302, 0.2660716275599459, 0.13454252387013313, 0.17691282667126698, 0.4079929278904269, 0.00929018772839517, -0.5618903650939794, -0.16326998876101345, 0.04900370823498695, 0.045928110828970116, -0.11331907763414158, 0.3170100667164974, 0.0069215356954751665, -0.04943544049286197, -0.22385849046266892, -0.011441851889649777, 0.01633064148235381, -0.17425249132922246, 0.04844510926589933, -0.03393105817296609, -0.22410863076425158, -0.04154862168148621, -0.20496855593889612, -0.1576214311585251, -0.3220285110291099, -0.347613797345803, 0.13134628351199798, -0.29173585255889395, 0.11559263473249172, -0.370843021099199, -0.6115532442841696, -0.5237674949919766, -0.7176106215512927, -0.7910583769932291, -0.7538024103471858, -0.8758176870034091, -0.9066508194603784, -1.2845315872314522, -1.282723325921669, -1.7016645756372457],
[-0.11470728101730897, -0.35362504772670966, -0.09217357364265422, -0.5874257799084411, 0.13421003115630897, 0.3369011793639957, -0.06557395048451432, -0.2719472257180659, -0.20763468920214098, -0.28687420516387735, -0.4354717382175891, 0.16269508648181485, 0.26123943508759795, 0.18284867223567658, -0.01936903216450263, -0.40206487494208976, 0.05832992406413027, -0.5108920475698748, 0.21377482552497412, -0.1871830746474656, 0.023185945280758086, -0.19913837092546158, 0.19770483667663988, -0.06285509953709299, -0.12016945976451317, 0.01883290505303511, -0.19571888430461606, -0.14552286155862945, -0.2885587890398185, -0.29934335011205776, 0.03471487466700511, 0.6665731421895262, -0.3174939612073475, -0.706599501326991, -0.3626145836000263, 0.13650936325132698, 0.2293811733469434, 0.1762398014022819, -2.2083752549026783, -0.9793146089107896, 0.08606824142896624, -0.47811913157722685],
[0.8157509790950958, 0.8246901377309052, 0.9760273609961088, -1.2797618904646264, -1.2014696747939977, 0.02725660153625966, -0.32136892688614876, 0.5276802404094206, 0.5477416004751039, 0.46708495087663676, -0.5817191325130875, -0.6435104976918296, -0.16190240514783782, -0.05021754225406848, 0.20031271834724707, 0.035700136114337734, -0.095286502602099, -0.6296723800623257, -0.8497097343946572, 0.13230012353174447, -0.21883242384618498, -0.5278385805749838, -0.612755335429185, -0.6661775079439248, -1.6602546071575295, -1.2617699733369647, -0.4589170423017514, 0.4737988332075954, -1.0256842016484622, -0.6812789057821336, -1.0858703441090929, 0.4768436480082213, -0.30888811291653273, -1.100340384733062, 0.14242468772185662, -3.3749832903458126, -2.8521765647448585, -2.9392991778702924, -3.4089729948072125, -3.017475519326098, -3.256638715844981, -2.8399100121998626],
[0.11621495612712333, 0.029256599559861766, 0.12554404497589733, -0.04612958287594163, -0.09613772941437383, -0.18246937263900662, 0.14875349358591705, -0.03159140438997045, -0.03526835601795762, -0.12423137755438547, -0.23371816229559766, -0.2749421947089281, 0.06956537827983249, -0.07913989372865317, -0.27762583472361907, -0.11803557166201963, -0.038633164887636205, -0.39453188356580965, -0.14376251256256992, -0.14375304899204766, -0.11968887850655403, -0.2228934869203999, -0.36263759839504894, -0.23781759164479524, -0.7067014273871659, -0.33443542271091264, -0.37489786239211115, -0.32305420213027725, -0.5301441440072955, -0.4130573534803521, -0.42141226242703206, -0.06745156701920414, -0.41180316305065856, -0.40054279682674726, -0.7277294362757873, -1.6199245462268124, -0.6978879641874218, -0.9288347557977615, -1.4840695556509464, -0.9174779670360815, -0.8640156537983165, -1.473396537003175],
[0.32278038396809755, -0.17010756383890796, -0.07999380796765077, 0.007221941010212006, 0.33861668711898374, -0.13347419852404654, -0.022736121366277087, 0.28616031728513575, -0.2052087508589545, -0.2194420724721743, 0.024324763888522737, 0.1088135810161974, -0.3590432918297202, 0.08179792837339182, 0.05494264990219738, -0.18788659517277784, -0.21981878946360775, -0.053592271727407374, -0.19455251797158252, -0.28465465824486574, -0.26332152186199154, -0.459088634834725, -0.6046723384438405, -0.3720498966465965, -0.40987435808248973, -0.2849769196137446, -0.5695779597064214, -0.15345050804273597, -0.6222331961828219, -0.29803473725376445, -0.14719788955990237, -0.6373548694344647, -0.4455922725241697, 0.1136775138657837, -0.46187877687683576, -1.9906354760925626, -1.3272862143348545, -1.4016021814942485, -0.9703183118469788, -1.3243718680096548, -1.4426840752396597, -1.4516564481029244],
[0.12660662287743948, -0.24462413893612486, -0.31053723616261397, 0.10461722093021819, 0.00954274025921148, 0.1819510466946534, -0.1306570223377324, -0.1742807964620335, -0.20715769947197918, -0.5855091803838688, -0.38260735951178554, -0.14828271078020325, -0.06961579926368523, -0.3081260021386004, -0.21116738835402216, -0.14928789611905316, -0.5075042885065671, -0.43744145927508754, 0.07777652242839629, -0.07693968612010146, -0.23722324086345953, -0.4608078339990089, -0.16920901554811435, -0.5185772303936038, -0.5062980426590479, -0.026684497144626392, -0.1659532479381362, -0.3480909181919746, -0.5168533751350012, -0.42643105667621295, -0.39766462947629583, -0.7356150658082917, -0.29419478976952573, -0.21597255336306237, -0.7373970372179283, -1.979120635612064, -1.4934295191136058, -1.7749455874787639, -2.1695079252762697, -2.0100374233909313, -1.4203502440872529, -2.215435196127283],
[0.20582774520324373, 0.027611499626923598, 0.248362129835096, 0.2823609238612, -0.029856092662217944, -0.13434104138298078, 0.16393159115879594, 0.1614436281760561, -0.008433362700439942, 0.050355115623645315, 0.1749443921269425, 0.06513979984345736, -0.11823530267948787, 0.07230656575600565, 0.1309510824022141, -0.23243381550435907, -0.07336116527460344, -0.06784266276551598, -0.23767799896288813, -0.24649670289572478, -0.0607610235597086, -0.24100577759250816, -0.5340435086879899, -0.19801010911258618, -0.30530365563195994, -0.473789728968158, -0.5201007437277357, -0.20479618454306892, -0.43815414306071293, -0.24625214100079604, -0.5214103554060842, -0.5454735810439792, -0.394386571048785, -0.1350402108998929, -0.5073680247668514, -1.7534958239787035, -1.2138875527412034, -1.384085068837026, -1.5071726714036047, -1.316857256418706, -1.260000653857741, -1.4689204306945303],
[-0.07484458368919203, 5.897471424274778, -3.3289318575972526, 2.8957838300969803, 0.7977152479888322, -4.602314535988241, 6.803282677881078, 1.114380175851109, 4.168648062530696, -0.5423271961192712, -0.10122011978646127, -0.24596814436900172, -0.5978785660942684, 3.153350029224356, 1.006250877359115, 0.6017970335518723, -0.3368227473182097, 0.6206984056056126, 0.43913729846236754, -0.9645096447149992, 1.0039659165922055, -1.7864943946064762, -0.40012152326026523, -0.6935773132105607, -0.40097603967887474, -0.46547790454512367, -3.461645315362651, -0.49045428247449774, -0.49112233682844225, -1.5495113069848292, 2.1904681190108297, -1.2028167606756686, 1.3821544818250675, 3.1323558906262914, -1.67163594045486, -0.32865666478159283, -3.3339789796288652, -2.0012926976324734, 0.23756870536099964, -1.1365874213855016, -0.5598024417548745, -3.9429478822692534],
[0.08483114318177545, 0.18903505841929552, -0.40299375653814135, -0.48193039140825317, 0.21479460885826698, -0.18255053774862992, -0.13776945912882418, 0.09838603003788135, 0.10261854769204257, -0.560369595814125, -0.5022364756388685, -0.146974725249779, -0.10224463962970338, -0.3339553679564566, 0.06409278226356803, 0.14396210977474272, -0.3929934450297438, -0.4909262393484382, -0.04183462740043688, -0.29789537557593876, -0.4668374607755278, -0.17511520703194203, -0.0910914395603033, -0.4975699535720463, -0.3288625417703187, -0.3550423281921908, -0.3613108070022456, -0.4855289381415102, -0.56732183651199, -0.5752741319646175, -0.3139869621958956, -0.2756279288400774, -0.4935642132146542, -0.43996142041360337, -0.6672854183666831, -1.7295268335339913, -1.1056338503131942, -0.7510468271661537, -1.6282683094874106, -0.8649467768297836, -1.0318437246734478, -0.9870259677527884],
[0.9964876871440672, -0.27994496563432897, -0.2735103794970629, 1.1129255376811882, 3.793069831564496, 1.7604534463131254, -2.5221373382672923, 0.2521804179364076, -0.6928337654775523, -0.5854396623697068, 1.1474412885393386, 1.0772664817946813, -0.3978248666326584, -0.4717213835367939, -0.05664212373849926, -0.24797185686461826, -0.27319900857959173, 0.32058959834922307, -0.0659073181573084, 0.45779213679301767, -1.3834517218975289, -0.42205019818047884, -1.095808151633786, -0.3280347026256412, 0.40537912070412363, 1.2073576329968057, -1.9005043082947224, -0.5062546000849341, -1.3293719561938402, 0.02487276291640912, -0.39836018497554776, -0.8991606736318052, -0.885044440110976, -2.3305171597005168, -0.20027191426928945, -1.4281618421044548, -1.259689584066978, -2.594240251847382, 0.1448041657189999, -2.950849596109654, -3.4859158193530706, -1.2309707239504128],
[-0.1674709024396409, -0.2407096326721345, 0.2048222133217079, -0.17775019404948675, 0.18725442010214238, 0.18272088960733082, 0.10785078616857018, 0.022006428113963073, -0.29025584302979046, 0.0612873562884691, -0.3570864122442428, 0.09030447062011994, 0.09899756484866468, 0.06861500284466239, 0.10309854978432477, -0.22256386143926243, 0.19182587589735656, -0.12748437636796797, 0.18479680501472787, -0.247441975912213, 0.04453611773521361, -0.29957811583908406, -0.1888913069536201, -0.17766073973004587, -0.22880361097215354, -0.0760005970297433, -0.2535844751183295, -0.30233455483216665, -0.7027780421522768, -0.4737645775859888, -0.473675854183606, -0.30260150902473554, -0.5350570122047537, -0.5758926474732194, -0.8773294995003426, -2.8812213479012634, -2.5065216394944856, -2.7630374194209915, -3.0052841096198284, -2.9279498859718354, -2.595484142282725, -2.914028174847895],
[0.044866943730851386, -0.3170337292578387, 0.013797769245598505, -0.10140346757606129, 0.37407251514345646, 0.3401425001261267, -0.23116635430559823, -0.1055085140207429, -0.15470175355891697, -0.13595253278368832, -0.28832920722096284, 0.09792214810918258, -0.11096999578031327, -0.20966115246002467, -0.13490265906650173, -0.15526001321978097, -0.27344570020162806, -0.18460455053343958, 0.25110236107678247, -0.13055471917283623, -0.24375394496370476, -0.17342732187361218, -0.18904925125635286, -0.30845631919531485, -0.2892339859404512, -0.12693218842014492, -0.05374631471818311, -0.17836948799813143, -0.6623409246047436, -0.40489969610815235, -0.48021552375900783, -0.12772227010462403, -0.3751273008196629, -0.4104655481642579, -0.9422931313757805, -3.0521880205565637, -2.6731282062382853, -2.809152378551741, -3.1093156879809376, -3.1736658791755534, -2.8065338869977943, -3.018668017002595],
[-0.03472802901634093, 0.02917656401771934, -0.3015973769691348, -0.04030652376726945, 0.8684262449391226, -0.07709014208034674, 0.4024756146041799, -0.22798081637824152, -0.42098027286933243, -0.23688848952698185, 0.038531114747073825, 0.6679849025051622, -0.35397833488515695, -0.08495377176665972, -0.2218813272817162, -0.6692669484286294, -0.07910284589822185, -0.11561620856483022, 0.9983965192272191, 0.1388387277569125, -0.1039269306337345, -0.2345294385860682, -0.1830869077709699, -0.07851123380956804, -0.06405671688263968, 0.02680824769988289, -0.10863059663771242, -0.028658206627529853, -0.7097358243353722, -0.23629938431257402, -0.18481578566848625, -0.006424507951539287, -0.18224224017061, 0.344625062558929, -0.7269166906788185, -0.6817326943265095, 0.5506554890634207, -1.2162491793821548, -1.15732048462114, -0.09438900593848193, -0.978745991818758, -2.0940255327773474],
[-0.06554078411759413, 0.12690121882688069, -0.022649689581131872, 0.05956332583960549, 0.017812419213493618, -0.03988346256038213, -0.015015953439214427, -0.19460814918905456, -0.16043324748578253, -0.26973557519137337, -0.18716759854691487, -0.0009015450034742734, 0.0275617834124011, 0.01911972626339448, -0.047578106291744894, -0.230122662279872, -0.23669516831388313, -0.32438446553856104, -0.2796584143605587, -0.0817901322261166, -0.11882664267635325, -0.3291734343061764, -0.45291521887648867, -0.277046885557649, -0.2792812904911025, -0.17574405970750598, -0.5118892359269974, -0.3206110596664619, -0.6697995733851063, -0.43909781566164524, -0.622578496904367, -0.36053194417215834, -0.44560356082795716, -0.22643089781608952, -0.6776576263794151, -1.5729191419032231, -1.2448080063300866, -1.3533664456828567, -1.794856644685944, -1.2239041753603364, -1.5475042773189707, -1.676050709821811],
[-0.5300963837830234, -0.4323278091393722, -0.4276985735998445, -0.3590155078554396, -0.15485506761724252, -0.29675683054251023, -0.22435202894609943, -1.2183097937623522, -0.4669052563320209, -0.8631263922460461, -0.3247592317849791, -0.5455710799745453, 0.15189177820298558, -0.5717975275574142, -1.2695676246619105, -1.905966247008077, -0.48705891950729113, -1.325575837774467, -0.7895158666643524, -2.1456341603819196, -0.3982258912657394, 2.0775354272449826, 2.228683193123603, 2.85444371516643, -1.3976405997752135, 0.515628294517317, 2.925603305647719, 1.5495748479427718, -1.2299044286808727, -1.0770748057608774, 0.013967264312137876, -4.846924707792362, 1.3074957002827365, -0.9748272482235614, 0.2857551881383555, -2.7531848763769635, -3.1674525944079535, -1.6609250527687331, 1.2552447980477965, -1.0115231289793623, -4.338066428521645, -2.4927981293281523],
[-0.10270179368835725, 0.33603710358415634, -0.029743451171120934, 0.07442723338866307, -0.009680129512758763, -0.13993778211451272, -0.03186501740386729, -0.23932670803951722, 0.08159655269774407, 0.036433337638473645, -0.20063452330964923, -0.2898896320761882, -0.10275445819250886, -0.19357238233904647, -0.23772732356697443, -0.19638820675315824, -0.24394779258926932, 0.09551697837703951, -0.25316971221239487, -0.12410442054256911, -0.0117495699179998, -0.13566855261606917, -0.2769210410918309, -0.3079322490968512, -0.30228650889206315, -0.09672966152717612, -0.38555265526563803, -0.23042860237977525, -0.7597174584791053, -0.5349265815070602, -0.3118203575820214, -0.4575494336801607, -0.3376986620926405, -0.3913074029836707, -0.762164083762189, -2.505712822613976, -2.2114232520606287, -2.715953412645856, -2.544820633766681, -2.527958890711248, -2.32457672366989, -2.4755463570121994],
[-1.0286325521655149, 0.35186093968324406, 0.4750614849489824, 1.1470098145170557, 1.1103883520067164, 0.30931133495978885, -0.36150136781250436, -0.5351091821621471, 0.11801975802693335, -0.06738718629463611, 0.44795594149509693, 0.570276888293167, -0.0883202194908132, -0.0176970467533818, -0.4149766171915706, -0.04253288223589385, -0.4852250937949433, 0.23019211790394553, 0.2275146515207629, -0.33345963722639593, -0.2225467449333465, 0.28957981090764856, 0.1868994101373807, -0.39140328939840213, -0.3704495141400885, -0.4742024998499371, -0.5695981920574968, -0.01084978484172771, -0.3654450895071777, -0.4177239824639062, -0.6409341249119607, -0.5702450443325751, -0.6947334942738556, -0.5493732356179405, -0.8278151606500047, -2.789595412398874, -2.610021059509496, -3.3410111567653673, -2.626006045170235, -3.2916953479132367, -2.7809507339142994, -3.229159977573498],
[-0.006688827011555087, 0.46347935955384273, -0.08980223478752827, 0.053482706397189876, 0.2549096806964834, -0.24046387205450692, 0.46274224490418087, -0.008612326693527766, -0.18853153344657975, -0.05872064999606434, 0.0793360682094574, -0.05155045802586661, -0.3093967726876023, 0.15003239688487474, -0.24728729591781626, -0.19012142385038244, -0.3516585709884277, 0.05491044373210839, -0.10096609595280917, -0.33588036261243664, 0.08614615511224401, -0.3240683410343306, -0.3411073129212387, -0.5910923238035715, -0.4770152659599383, -0.038082512231918705, -0.5821215735367715, -0.45269585899550085, -0.8474797614594702, -0.32440285249480644, -0.548833544460171, -0.29017644103552526, -0.5387259927392152, -0.3252247393048143, -0.7258653370120556, -2.309828042555265, -2.1084764855722478, -2.427384248071924, -2.608935936931154, -2.3427085197666155, -2.4007924329886134, -2.708742698170055],
[0.002711943587437131, 0.2545077533392007, -0.2460729832416088, 0.00964346901087342, 0.1536216389928008, -0.012849906558753278, -0.18582787122871522, -0.1390248743569531, -0.1320533956790603, -0.13110124614988414, -0.15587498361480892, -0.160650321472226, -0.0010474361744083399, -0.262967291230811, -0.10887504929305179, -0.26567771026692005, -0.2914092948707461, -0.19445484543596736, 0.0118868074000999, -0.21444467803681494, 0.23899521751161595, -0.23256194819290385, -0.1451070263266417, 0.09385533596252739, 0.348575166137442, -0.10149471571740198, -0.20018249279760955, -0.26390452575022744, -0.27250120749742773, -0.39182104512884514, 0.45899639535895653, -0.400701185086663, 0.0934997010634848, 0.396049183642046, -0.8790369871124295, -1.0765320912595657, -0.9597232690871371, 0.9945781827329442, -0.28384068607315366, -0.5758290540874699, -0.1288373810734049, -0.3451700472501719],
[-0.12778048678922752, -0.1658809560184235, 0.04289947169929763, -0.1081788641390652, -0.02219694660458527, 0.21458405861202515, -0.09110208054464565, -0.09355624209137389, -0.41910208820060835, -0.15915297672007228, 0.009722209633496782, 0.01437313072973064, 0.09793772033573835, 0.0937090227361486, -0.1051888019299611, -0.059757094619265566, -0.04789111833612214, -0.5755069432723351, -0.6257652334304746, -0.08973881344703838, -0.22454936337567652, -0.017328436793144133, -0.27855147436996003, -0.29411095071638277, -0.5493691533202141, -0.8283730926516122, -0.34989901732199213, 0.061346523670484876, -0.8047679047034108, -0.6452542760520318, -0.6596492216559153, -0.10645225932319523, -0.6725411813976265, -0.8743397551589508, -0.8157069064596814, -2.703221790926943, -2.4877728024230046, -2.611671406348435, -2.987999559634411, -2.7506720033932606, -2.4732571821998097, -2.585312819048916],
[0.08164600868625345, 0.13048098069847686, -0.14831948016582155, -0.1844670861096677, 0.0020771503783107038, -0.13010827758025678, -0.11851942521134837, -0.30002496816328633, 0.06133506571256546, -0.37898746669869204, -0.3742995211266473, -0.14176484302719994, 0.08992058941275576, -0.1540795507266429, -0.3839214001635471, -0.7028035673710367, -0.07869642377391538, -0.798205878557403, -0.21510495390401344, -0.33309778204307805, -0.5256533484482371, 0.06452824702044926, 0.007128123914966183, -0.302420544679477, -0.45062266092596187, -0.3958342939553505, -0.33708675030970503, 0.2755529434686337, -0.175856721953842, -1.1884337816787174, -0.4372169574431072, 0.15447159508492567, -0.4022944235916309, -0.931242438657317, -0.15173441138893404, -2.141034178972963, -0.5694467231524122, -1.6185983636902326, -1.7840312332350587, -1.359740736309138, -1.363394283709041, -1.3311259244629008],
[-0.033537210836797726, -0.1264132016057463, 0.17903613047915554, -0.3756888722714399, 0.3877282697500678, -0.2611332848827832, -0.10957418367362254, -0.12199427076119766, -0.07411673402207748, -0.04090946788202098, -0.5263294456875867, 0.08763167625819623, -0.2103509301153987, 0.020253019782472827, -0.14450692444685737, -0.4523189703873688, 0.02241878141643129, -0.511828171201946, 0.045228848830643535, -0.5092969715452723, -0.06337686517278074, -0.6403660795771947, -0.592802051111822, 0.15635582951760982, -0.367118363691724, 0.41571028130542986, -0.5037285022481928, -0.2874741926011249, -0.3071736104697141, -0.728585339402237, -0.23999089082172967, -0.20680585458357142, -0.03780768679993316, -0.628076049692734, -0.40563095264736365, -0.8328820432637535, -0.4085974002058914, 0.48537606453539384, -2.1823019467472156, 0.10794011258529654, -0.2073117115416487, -1.1038189844581106],
[0.23431842176082568, -0.2571807603557187, -0.2196286120200673, 0.08171091206371828, 0.1673724291729648, 0.04116394172936554, 0.12300450977923723, 0.12902725803471127, -0.26134729145060215, -0.30007835750879347, -0.07384844254329967, -0.0801105562273881, 0.016019139757032334, -0.014837575650793784, -0.25973455386089406, -0.22296466704529083, -0.06537451678202613, -0.08203033412272141, 0.17177199702896218, -0.327262519873311, -0.12122112893038169, 0.05239994263482772, -0.4196182980638037, -0.3480015895522359, -0.2637178962825707, -0.18819525279291963, -0.46782405606808547, -0.44361333893904387, -0.4481869664033577, -0.4340084093041325, -0.18192455154505918, -0.2600652786694891, -0.47249293394714276, -0.6361547073955895, -0.7177740369367506, -1.3658014214377039, -1.4602851908175078, -1.3305013141381417, -1.5050207034049208, -1.5890479549227323, -1.2759353607999397, -1.8670120975511506],
[-0.02078806050573598, -0.39797794545598864, -0.8974041561557102, 0.44493255183569946, -0.02303016188532469, 1.04771078511687, 0.638918022863003, 0.20443439309456768, -0.6382034479047209, -0.889388300047784, 0.11106708510053555, 0.04003738740954899, 0.22325387277679684, 0.7224996515688862, -0.19907609981414373, 0.056013120664277556, -0.11283296871173905, 0.280937641626878, -0.32320330821671833, -0.019429517344093962, 0.05160178673008161, -1.0968979696856287, -0.9958149309679897, -0.26189452705893557, -0.5779343443511133, -0.6485501436729886, -0.47440525713802384, -0.26362526517749324, 0.5608881727523822, -0.4447697399500101, 1.3985539945209808, -0.339746955714622, -0.39055524758585347, -0.9361196533157399, -1.040545412639002, -0.1320154798729344, -1.105153025995388, -1.786993415548102, 0.8537827053859451, -2.5054293550978475, -2.2529074248556227, -2.6506749428482683],
[0.16822820198310628, -0.11721843847475316, 0.046797150550867556, -0.13415756518311878, 0.23079434294118045, 0.10741067591266316, -0.07471605693862518, 0.06234048513488375, -0.1869450300027459, 0.08892361569840732, -0.20873826654982483, 0.2357958824926688, -0.12813399301258788, 0.2510730544241695, 0.37418981041678673, 0.5050775739041745, 0.15480913574651872, -0.09440444624283575, 0.34700704235962077, -0.2088014654578045, 0.004822475418930515, -0.015544448089373615, 0.19852052863481312, 0.5931934370688433, 0.04608117977733943, -0.294536667338356, -0.5300885097498552, -0.29536919711698595, 0.9965553152846152, 0.06872904083647756, 0.4878192027087766, -0.04986951845788665, 0.25710392692058137, -0.8471265129913184, -0.5266089601259261, 0.5447348559464453, 0.32725394042617445, 0.393932021049673, -0.7270086942119381, -0.9309521811169501, -1.3646932396413336, -1.041662249027099],
[0.20183365651656693, -0.03249706847185701, -0.041355953661045275, -0.6252276985646801, -0.74085316687481, 0.41043237501705127, -0.11336062105910433, -0.13788796527260225, 0.01767356523226802, 0.18500408803701074, -0.3157552738998561, -0.4774872940958943, 0.7516554287258652, 0.133000030465825, -0.31360267284355114, 0.02741166518062024, -0.2549904252591943, -0.45137683429358477, -0.5456207563979895, -0.35251274011171446, -2.0702470699197497, -0.6309853121331535, -0.36334121535144154, -1.0841018258632662, 0.6424359315404302, -0.9764484634350654, 0.16283152025138145, 1.7961838402191008, -0.9276234965235899, -0.6163123306672401, -1.599281550711555, 0.3487765079326009, -1.8542730895826873, -0.5581076485938119, 1.0359409720349106, -1.3910407514870524, -1.3005578928873176, -1.4559289430403475, -2.4610322572862935, -2.4727209791256146, -1.8408600831253805, 1.5400272931839014],
[0.36692621402824294, -0.30002258141722554, 0.3045543195258802, 0.3182027596533277, -0.026369356759751793, -0.19352366524673012, 0.22769577676744948, 0.09189469003000737, -0.22431318890628968, 0.2894911309705567, 0.0401302472349237, 0.014768491173352992, -0.255544848057901, -0.1303768726763952, 0.046871524831352734, -0.2704764826288213, 0.040711966588628176, -0.15376428158289615, -0.16184278407582262, -0.26344367181156514, 0.1544587346514287, -0.28088639370065444, -0.4579966510438463, -0.4476313559415836, -0.47171386301327195, 0.25121082606325934, -0.729591395614343, -0.2246022882693272, -0.8529375121947828, -0.6651645618292629, -0.7852716814063208, -0.3067759311131516, 0.006996416496152811, -0.4790751939382114, -0.15866280901361504, -2.139983829971149, -1.3252828325489436, -1.6724357881053704, -2.2228191796019168, -0.9223037357529342, -1.1680034881875416, -1.113971413799865],
[-0.15044283208159992, -0.24129100098778122, -0.517938329780244, 0.021205006308021686, 0.563683691604265, -0.05739451811390354, 0.8314381803430106, -0.11182307924261733, -0.1876813248925165, -0.30934838628149797, 0.10308229602896476, 0.4319048355430384, -0.08470825652443514, 0.5201334230715202, -0.4053702238588136, -0.01979705241765834, 0.22936508772137087, 0.2578795765922686, 0.3374240980940696, -0.06435800321467497, 0.30424976471668647, -0.2222463984310755, -0.5471803913842906, 0.008482774957747642, -0.4873900605504722, -0.03152470604553802, -0.3139587784690813, -0.0975171420373966, -0.160389269174939, -0.18706367624288656, 0.24768914036847656, -0.13861258403017976, -0.46767026274999407, -0.6224881161561977, -0.6099176053097831, 0.2577739136260904, -0.4107354466522464, -0.6130370623889386, -0.7893406563668846, -0.5898446271400186, -1.2665933336165602, -2.1732105792781478],
[0.3760225442627033, -0.2140861661723924, -0.35918133593211615, -0.5056722805787243, -0.37081303118039244, -0.00927317229588215, 0.032535255399412814, -0.052410180371447616, -0.23522273488987006, -0.20011244498992012, -0.6273963274888534, -0.2292212481789846, -0.03915405315873407, -0.2701007622378319, -0.18396374445710212, -0.6108912475086367, -0.2754280697329018, -0.8113366126401028, -0.16282003205858991, -0.5197212698089556, -0.19933704050575782, -0.5585813183790306, -0.5548555074615856, 0.3946220574416875, -0.4231044697245628, -0.182683198248362, -0.3872271266264958, -0.26828160437201776, -0.7666756596003762, -0.9256706068063822, -0.30938153673886215, -0.06679215848502003, -0.3706521623518438, -0.72730040896041, -0.7362208576150111, -1.8206245221450126, -0.713712447288782, -0.7555708696582986, -1.855585302561881, -1.480224967981747, -0.07558058676689931, -1.658037062406927],
[0.5417032091169013, -0.2597441321996567, 0.2375885443018634, -0.5519805614085151, -0.3111050190809463, 0.23271354003999947, 0.4184907367406822, 0.38987382638743173, -0.027154747875184647, 0.15652066793062241, -0.22087276598487834, 0.07536636215508913, 0.49694387210301655, 0.17558205342757346, 0.5245798519938196, 0.0668955716515241, -0.1633695597448986, -0.687074697372038, -0.4993237420103969, -0.26306432098484406, -0.21110457444656464, -0.1068747126105244, 0.05490107685264842, -0.6206427916535249, -0.28878948630649487, -0.40855961017515946, -0.09688617399802898, 0.5128711181422742, -0.4016762057191623, -0.22961228185401614, -0.8624376420112925, 0.4914001446237596, -0.509552654140457, -0.6093030523810676, 0.36023650028974097, -1.4536540482081952, -0.8027710943847168, -1.3138301517962965, -1.5101397926735283, -0.7953429170173822, -0.8229448841985315, -0.6316171953451087],
[1.672423228234801, -4.58174627283788, 3.238686533455195, -2.0283278942865604, -4.772116982829872, 1.5811799666344357, 3.4914633123529097, -0.8241313231612568, -0.03309613722856903, 0.4773768153205158, -1.5812542021493432, -1.1613114791197174, 1.8159324653274542, 3.5392156035770106, -0.37422094067058836, -1.5341570249736376, 1.324204733343717, 0.05235558956357372, -2.0095732108524724, 4.315661530801939, 2.126104532135577, -0.0761690338525492, 0.5895990590245588, -2.941227794328847, 1.8749912740649615, -0.9307189341373707, 2.0839394122024393, -0.7695063640002258, -0.6979246273434001, 3.061542124759974, -1.6808588797849848, 0.8340936435427596, 1.845257720847155, -1.6877071323662338, -1.8265321942792516, -1.0987915340603227, 0.8433456079745152, -1.8469109483449713, 0.22179845503188078, 1.3428108439755393, -0.5577584497158881, -1.008559924474312],
[-0.8777656019580486, 0.45366442658318634, 0.22901750085090758, 1.1605986765075935, 0.7910300344139368, -1.504140010474775, 1.2131835827170314, -0.35765425965661657, -0.010366045047174881, -0.0860360954285244, 0.36331659375980324, 0.15959587743824294, -0.7536338523659, 0.9802216283415438, -0.27164290498701843, -0.15715021514519947, -0.2822779404114972, 0.4785815401396977, 0.13189422081336857, -1.2054737617055442, 0.6101326981181718, -0.11658611225232296, -0.467797014492479, 0.254822214029558, 0.4808951133526388, -0.42734483863794664, -1.3604589472973228, -0.6687783514417407, -0.567126415998904, -0.6830252823399066, -0.5397897505105712, -0.7830293634035922, -0.6023292652806375, -0.5098634906203147, -1.326182984258002, -2.8416766837556504, -2.6850653891981398, -2.699468547550348, -3.0691583670349423, -2.7867985181418544, -2.7978779504272127, -3.043733374213797],
[-0.11994865308544082, 0.10205757206856123, -0.03143021381483657, -0.18191083446308365, -0.0019466089813677, -0.09752014355121816, 0.154667656969711, -0.0569396425677088, 0.12694011575545877, -0.07352506840743049, -0.3404991911885858, -0.36480601769372495, -0.04651285744694027, 0.07864265726469595, -0.05547110187720687, -0.07331116687339585, -0.048092425827383975, -0.2722595104102711, -0.42866581662769776, -0.11831063662344865, -0.04999784882916509, -0.0416349875844179, -0.250278170328877, -0.22210893165407888, -0.6243478762759123, -0.38012604107493786, -0.33759836662857606, -0.12461930895285162, -0.5582162280873204, -0.531173906696585, -0.1949691792652764, 0.03574501696565357, -0.5499271570453769, -0.19814496990951488, -0.39174007232691843, -0.8043200567341805, -1.0309626750580387, -1.0178302474686092, -1.5825198950310608, -0.7132445753834004, -0.7620092518924121, -1.157542451721261],
[-0.18858500030911332, 0.09423335336324933, -0.12657332712625924, 0.11285517480567105, -0.059551003289330814, -0.02274795151571172, 0.004520055292872384, -0.06752727862841729, -0.12476866402806362, -0.36805727688409623, -0.17645578740393494, 0.0042195721069851104, -0.23300406395040374, -0.15956653244197466, -0.20327734562407926, -0.04597017214983747, -0.4377342999138832, -0.13221977314877634, -0.014111303843120707, -0.4195821188830829, -0.22207539384555208, -0.3326805683789308, -0.1616718637729109, -0.2694862265016851, -0.05519232616097038, -0.2207484110192644, -0.68117592674436, -0.11022985588647026, -0.6325901566510189, -0.41278224474253683, -0.48197763567756763, -0.44645333621405653, -0.654881201206947, -0.6841195303912184, -0.7572029088700913, -1.5055560330293873, -1.2325941971397212, -1.8348859653152014, -1.8756537650499692, -1.9230223402733868, -1.7532181328802747, -1.8956984386283844],
[-0.08361773059306252, -0.32980927005444116, -0.10036362158576985, -0.029706342959614693, -0.022088650676400586, 0.17035599122467257, 0.23669985110888878, 0.05031283449829135, -0.17869573386908905, -0.11911445139350658, 0.19320272672801292, 0.10033131559464546, -0.010757870022113591, 0.09443825295625852, -0.2113252164573425, -0.2092718087068344, -0.20807586633571404, -0.12438297325374083, -0.065271373590939, -0.16691628583233326, -0.15102483740518083, -0.013711727808671139, -0.25341624889676184, -0.24706338036584685, -0.1373134969847958, -0.3752641154220549, -0.5129412111909389, -0.23973279719556292, -0.38071246502647815, -0.5551365908865792, 0.019785575550536283, -0.19650876714560728, -0.4373997253501833, -0.8725008841335937, -0.6903466817359025, -1.115648469909864, -1.3703066089986533, -1.3328265717681966, -1.3778317630365422, -1.580199718137268, -1.3509423602455384, -1.63772747026157],
[-0.15144356849478277, 0.7809456789034296, -0.04259805734471148, 0.7906560545236321, -1.2643111928509727, 0.721930753939881, 0.5245775447369516, -0.03852416066065632, 0.3628840911821149, -0.13060746903659376, 0.22271960721408898, -0.7396461466958427, 0.145345184175854, 0.1909428340913963, -0.08909878357875058, 0.14719461133879103, -0.2713231626101298, 0.6471497358632767, -0.7895497468353482, 0.03618718436535301, -0.3596592325145462, -0.1397766592121906, -0.19293076037639678, -0.5722305894732541, 0.0856484576292622, -1.0891679833525607, -0.7048137120355324, -0.32025386393429245, -0.8637027684705039, -1.0209422438058833, -0.13045334358913824, -0.14694644873057106, -0.48932400739792403, -1.2953441496351268, -0.870849070505371, -3.3852989691489954, -3.049288417494888, -3.445255155906083, -2.567577864374295, -3.6614508561495533, -3.2569622728164362, -3.602038956418579],
[0.6132548133198842, -0.46605826061235384, -0.7095817812407621, -0.09345076950927289, 0.9301227771867604, 0.24064828993066792, -0.10448815726694384, 0.1461988698265634, -0.4522728533324736, -0.6424415939341026, -0.28922668857042355, 0.6480102071043224, 0.2544760897695877, -0.05749432003916663, -0.012948690256291117, -0.7242111114961319, -0.47276886234471294, -0.41201499782363227, 0.2597552729371028, -0.19228483445412345, 0.18557356591274524, -0.26493486425768975, -0.4687058953547723, -0.8213923877071135, -0.12187039933001477, 0.047277702094865065, -0.02667001184358425, 0.052544226012244184, -0.46556196253450194, -0.6738979893475169, -0.38302113998547027, -0.3558972752821949, -0.7395187622256565, -0.4370318908342905, -0.7085100712544969, -2.9828482653192014, -2.2995193684756807, -2.6010923567796254, -2.343699885313928, -2.6329524591763502, -2.569215080426405, -2.8672827522284297],
[3.172940600470381, 6.819170607538103, -5.461642029567879, -1.9724499148462387, 3.5619746122386946, 5.6653308673554506, 0.9877178769503424, -0.18903382496758372, 2.145807415505388, -4.019627554271012, -3.065361176776966, 2.3206352138594095, 1.7982719644848524, 1.4206212371566873, 0.5095028787445887, 0.7279123665554607, -1.0599415499190379, -0.8679902155812507, -0.4485810951904029, -0.8900612015590276, 0.5564817647373452, 1.4339203199140667, -0.04712559247008706, 0.22976969799978128, -3.559463104038347, 0.16515527940099425, -1.7864982190140204, -0.8761317067174039, -0.7343187128166365, -1.9802235562336346, 4.749604571446465, -0.37124698376348975, -0.6409815654670113, -1.8025972402295167, -1.8174038460707975, 0.36550886736899385, -4.851018990596929, -1.929682138186446, 0.3105233898963407, -0.852665573748077, -3.30362216776292, -1.503290201564665],
[-0.5030751115978668, 0.18542592651225098, 0.08945099028795916, -0.060234999873865006, 0.13306898011397972, -0.04860527964207302, 0.1834955271641552, -0.38472130328550413, -0.2621301014333918, -0.4516775409890386, 0.2524082729783874, -0.16541280339515205, -0.2447582262648939, 0.07353951683835373, 0.19975690031807253, -0.5192349566799308, -0.43268740357378155, 0.060038929873807836, -0.2636520227210782, -0.32619640389846044, 0.04757411076235131, -0.611660212431473, 1.090402520706962, 0.1002827555518154, -0.10767663304662074, -0.5989495426097857, -0.3544968954896457, -0.2669980820406147, -0.8696594385354335, 1.1798054387421035, -0.6545715323650046, -1.3765024735450406, -0.5661096177196909, 0.05133888371023063, -0.41444673333641324, -1.9476378268583918, -0.9828572385053403, -2.458487886040321, -2.7494681509738803, -2.105424582748793, -1.6218288259105775, -1.7641375583493342],
[0.08303100636959383, -0.059420349293648686, -0.07572251038779121, -0.34384025075538865, -0.17114772026284114, -0.10192407918054185, -0.07136075783313811, -0.20813668650841902, -0.1302388650754462, 0.054014495131277984, -0.1837167539803095, -0.1618471727248506, -0.05409880977263019, -0.2362266837238583, -0.03289141424154408, -0.3068336361243669, 0.1569448617614308, -0.3424369343034005, -0.05138907720201926, -0.19670677312810123, -0.13308341529736079, -0.19548744183920266, -0.1491638075169601, 0.07627525322734298, -0.5812252819865824, -0.3964959655784075, -0.3084040916929852, -0.14684198101030077, -0.7798082605227086, -0.6110152198878018, -0.6018874294789659, 0.005616914836750583, -0.5033779900951679, -0.731683053375419, -0.5820192781382604, -1.2565276180631504, -1.278513254931936, -1.1134008734601033, -1.9867525179953605, -1.1619871424824444, -1.131465514068132, -1.0568428094054438],
[-0.01423778508274946, -0.15021093321927606, -0.003819121739683075, -0.25959449304467397, 0.06528389181324501, -0.03887606132643041, 0.08595754533403577, 0.08445635839784299, -0.14066389591260886, -0.3303681946418761, -0.12697416808316817, 0.16146251008396303, -0.0955077740650872, -0.16896620697300424, -0.6855059275103345, -0.061604555798767044, -0.32132032571101016, -0.5506875131659721, 0.23263295698842903, -0.4907884279209403, -0.22307527858840157, 0.41548369457922296, -0.5489960294626166, 0.04650565067203747, -0.1713089814441575, -0.25431937019833056, -0.25770771564596584, -0.1500126741110135, 0.6101158668163311, -0.276531940590969, -0.6793615065314331, 0.23583109918993278, -0.7674017882810303, -0.6733098996499127, -0.6244898237520923, 0.13174468671119682, -0.9931567038806354, -0.9803713031342449, -1.424074075495531, -0.639116417975549, 0.3009673269022826, -1.186843640189667],
[-0.5352764137836694, 0.49617485983316856, -0.12836712712029757, 0.4765403227860597, 0.15899311641234162, 0.061676730499593026, 0.12558519282002795, -0.09118115710139035, 0.045857020963361306, -0.27771686589701194, 0.29659372836845266, 0.09313662137637296, -0.1577037036077009, 0.07380030686300199, -0.20828175564301868, -0.009139263590248237, -0.20347339674107767, 0.3379144810711794, -0.007986409004314507, -0.1600562689105575, -0.0057999609463735904, -0.5000834909526006, 0.43081830809279864, 0.36763928253502315, -0.07766649119249609, -0.2349063392415532, -0.3346200081459793, -0.2728726104247818, -0.9560552995818513, 0.6317574680116929, -0.31251681056062297, -0.876037255842098, -0.015629762007816815, 0.309463807061799, -0.6819972700332234, -0.590054673473005, -0.20778505729052385, -1.4977094526216814, -1.4480415075408093, -1.4065163885489818, -1.0393216215117203, -1.6134473686128015],
[0.6436375998749965, -2.6931371457920643, 0.7290151829581776, 2.485593356586888, 5.813238069025601, 0.31228427165797323, -1.914394090296786, 0.5262308029912043, -1.4238967574172148, -0.5164518849022237, 0.38385003172104115, 1.3938774523833486, 0.5901282413776815, -0.6528823629097207, 0.2519800107245314, -0.9734753787187526, -0.2991106827187457, 1.2591241475046218, 0.9015796071556317, 0.8161684310690428, -1.7029742867108137, 0.6874468072720729, -0.7491361997110737, 0.06620829831404512, 0.2777334504133553, -0.2950744093876644, -0.128244442550649, -0.1954831487654459, -0.1867107928703499, 0.7096977653969926, -0.5452584049346024, -1.619783679066275, -1.4472985809133625, 0.8764580469503537, -0.3599968876186491, -3.5426417739235685, -3.0796238493565418, -3.1136132353807264, -3.3874735275562458, -4.064940697634112, -3.4586875955199403, -3.6694044663730523],
[-0.062213118691529734, -0.29397844784274263, -0.24434593167251545, -0.4345176252155828, -0.1705333939167274, 0.03827976148092629, 0.260248802044176, -0.1795813605657401, -0.23013722397191647, -0.10316777075595236, -0.23530941105329967, -0.09185876800634525, -0.14686633369739627, 0.3233741465536884, 0.0031470000147771626, -0.2571027634817779, 0.011619524894151658, -0.554811629845365, 0.08076132926972107, -0.29360754110511855, 0.06749146109840627, 0.013196258061560478, -0.14269997971698334, 0.022488303744452674, -0.30349612741817167, -0.3213295591701377, -0.4553234644425422, -0.21521074277905797, 0.0723470112431264, -0.665513332259114, -0.20641638721137198, 0.07437493687012635, -0.37321793631404604, -0.8023949218293083, -0.4101404170561907, -0.8604135777640634, 0.003031420906005261, -0.7224225033113104, -1.5266926716510778, -0.7458805574351406, -0.2193262040466436, -1.1957413044297067],
[-0.09899223304456935, 0.1664818413822352, -0.2718076835915095, 0.07637976740311853, 0.027722654150039244, 0.09914984867581993, 0.18762969583002076, -0.20921332428865677, 0.1332862471065486, -0.22410209900897135, 0.03154146522104683, 0.12065146108578188, -0.20322347349863945, 0.13287132517967185, 0.053482131015599364, -0.03693581319432765, -0.1283110698813646, 0.0880405219077509, -0.018561377736124495, -0.09796114771111382, -0.18805264347970982, -0.23840649821218585, -0.0903699515124264, -0.23542587487973324, -0.4215055274142722, -0.09086056879148197, -0.2896384745203554, -0.1491562693521976, -0.8197776402556677, -0.3422604728450906, -0.29566514786437237, -0.4761643839966759, -0.33350284116803514, -0.031683874921689975, -0.3965034073055457, -0.09661155995222126, 0.14036687209956805, -0.5229447706445, -0.797122887771025, -0.6606454443396421, -0.6920112952382103, -1.163439534892664],
[-0.14220847266542, -0.5327352148992514, -0.3746698758845963, -0.1412836966283123, 0.11123108543373032, -0.09622996606912361, -0.043181014509669664, 0.6401091441170675, -0.15066508804437964, -0.8429297982438404, -0.05320555661771826, -0.1902319193272829, -0.5994182803593656, -0.20207729969058402, -1.7668933036511654, -1.4157469470532296, -1.76958207181973, -1.1231448974950287, -1.7485993599337384, -0.4283991839022897, -1.4240018202804883, 1.8275385911114548, -2.2798826385155344, -2.0855384302177056, 2.5276235802561526, -1.8744009087548434, -1.7691896360918962, 0.7183927170464229, 0.9120311509924571, -1.4804762346069473, -1.3346697107137897, -0.07011428618148788, 0.06523740330925618, -0.9405383041577488, -1.1073416205193447, -2.9536561025413985, 1.2998044120906533, -0.8662959637056316, -1.541303260755565, -2.072874902305211, -1.0224851008411657, -2.1291761977076757],
[-0.05582669930163672, 0.04884750054278984, 0.04832247758463263, -0.11407569073915715, -0.20166519772697628, -0.1511500354241953, -0.1504782828408545, -0.08082739112324883, -0.14835248397157794, -0.07174108567534311, -0.17164602671721343, -0.2272136021772246, 0.11262793316967584, 0.04644227358801109, -0.2741952654687009, -0.2902879931191531, -0.3036298304848512, -0.389322661915893, -0.20445100865394628, -0.30893410299695195, -0.21675110758454313, 0.08374013889446169, -0.044624332394954606, -0.3109101613008298, -0.2523377781038115, -0.14567231211135565, -0.0820259958925684, 0.02027359254167094, -0.3762090975193754, -0.5763214763279242, -0.5539782126764811, -0.08422054511456196, -0.5257178561390878, -0.4059016064227266, -0.4191378187683083, -1.144602450606435, -0.7721701486999671, -0.9924795372147355, -1.6538427760470404, -1.0140818230656756, -0.6656931700167896, -0.8646889926553377],
[-0.04104364188770339, 0.005207383511415489, -0.07704001140304542, -0.2221586377698567, 0.031388694901804036, -0.00873013679038014, -0.1869665151280072, -0.07544794240658369, -0.1758295791518532, 0.36803533326678783, -0.443591958719768, -0.17114275814389324, -0.029022056934548485, 0.007286084555039131, 0.154569459843548, -0.3491923492167571, 0.5083905034891406, -0.28628583791385487, 0.01241715883379128, -0.05038450366664249, 0.2888990117448146, -0.07550643895778286, -0.4749887444913702, 0.8188574327011457, -0.47533429288664797, 0.5173486672991201, 0.3837856891022656, -0.22155844424480103, -0.1511984585199952, -0.4430650655752872, 0.8476856126817317, 0.382551044496405, 0.5042130672152925, -0.10420670495421215, 0.02692525777342234, -0.41998410701044675, -0.16346231664555597, 1.141341705573684, -1.3642574799246974, 0.3440349838548573, 0.3208149034935066, -0.02012595242049694],
[0.012534966486624381, -0.29185116037676445, -0.06469585414041097, -0.437874258705397, -0.08049278337331117, 0.12205470544586267, -0.0629070242916762, -0.1691115431209493, -0.11496109163801232, -0.2786034322735366, -0.4168340427761059, 0.03357244906834237, 0.03257862246734285, 0.06445847652803381, -0.2547546450736193, -0.5405902846186468, 0.10953072546294258, -0.6410565590639115, -0.14952698569172326, -0.5438091265334442, -0.004413340155850561, -0.3919189712329053, 0.015725023064843868, -0.09215423178986037, -0.637282226617401, 0.0072220625455032906, 0.006860487658875231, 0.027952731697073838, -0.7264379040371893, -0.5520948719972831, -0.6345585871186663, -0.064823703325353, -0.4762966767528419, -0.8131165957584339, -0.6423064921416528, -1.833358486139498, -1.7636724907368373, -0.6725742559341191, -2.9951228287902625, -1.6233845704520444, -1.5472768668293362, -2.103565378005134],
[1.2773098942797334, 0.42352581487656227, 0.44543022759967793, 1.8785408959498555, -1.2146330654886495, 0.5046776543871516, -0.3406120410161817, 0.8092947380113179, -0.2818802720930852, -0.08687320802697869, 0.5296440307814483, -0.7783618104653486, -0.010511466946627175, -0.49744519645745394, -0.1687890098708056, 0.4416014602472116, -0.703138985640836, -0.36679340908765584, -1.0883713337670056, -0.6934164455672024, -0.5845078254997228, -0.37038164597481266, -0.32982919438144526, -0.5909132093155409, -0.3227589875402057, -1.2583727301077772, -1.00447399834767, -0.46482520117034903, -0.817379071209554, -0.10595647784720016, -1.059902039410898, -0.07822547175932272, 0.4811785476916627, -1.0504968834659367, -0.5948237037462819, -2.524349795559554, -2.2471353627753725, -2.3606262924619217, -2.1824812115328167, -2.135069598210735, -2.233873577480832, -1.3243584700951636],
[2.01138123962993, 2.8552283958471256, 1.9913098780720195, 1.1693984872377619, -1.514786007989106, -1.6488756926096526, 5.014264224058746, 1.0921704433423103, -0.18298025132548845, 1.125591831918641, 0.4522170206098044, -1.6603850758918388, -1.0592986275600635, 2.011676355156953, 0.8677801304691841, 0.6663354490760562, -0.5054374984668111, -1.1424772018265308, -2.410439854577623, -2.127691559078218, 0.28534042408561267, 0.389211245494714, -1.3935368400660866, -0.11237834296706907, 1.428705322855059, -4.4453898558914355, -1.758143832944342, -0.8214440531636454, -0.5087123881256106, -0.3874580483298871, -1.1136253819112387, 0.6507187332364511, 1.2799203761207996, -1.189914250243456, -0.19510770784592868, -2.5088886648750837, -2.860105965993072, -2.33548062244189, -1.7085391419990337, -1.1711160490962524, -0.15005484226203336, -0.9098450555979191],
[-2.036929977415791, 1.5865191117527047, 3.6931762940986244, -2.9847473438741288, 1.9198871700616937, 1.1510223692100237, 1.3387821647248717, -1.093682888746202, 1.59591701598088, -0.1739370093086979, -0.9990548604345146, 0.7163025156526407, 0.6823428848757994, -0.09134965929423247, -1.5534656600586454, 2.7584117363034872, -0.8985482000025334, 0.03253336331976134, 0.9239884327774348, -0.42268134087477194, 0.75840408349841, 0.19049086713379818, 0.008866095368846222, -1.7681661202192704, 2.0184878730925937, -0.9124046376512811, -1.4175110894791363, -0.4577703925940638, -0.2457784329292294, 0.04222940234090181, -2.2540816214962023, 2.3133025597909485, -1.1886017982387571, -0.30446271515172546, -1.4524122092963758, -1.9297000520546215, 0.8541640057858665, -4.214628921254348, -5.7358635699871865, -2.8042288108084557, -2.4111144044786594, -0.9563826199341164],
[-0.35147287364045043, 0.039066787452204564, 0.09200378550840191, -0.3549530939685933, 0.004947193257713842, -0.023013690523710488, -0.01919275494966879, -0.31883303511411354, 0.17213246421568384, -0.02590565970526741, -0.3881207574839731, 0.09115613497583572, -0.05921690630817867, 0.07797589533915152, -0.17218001290119084, -0.18595574230376086, -0.033133616378999226, -0.2880715917113593, -0.16856003191583815, -0.12403403036712705, -0.0875703262536489, -0.23701970113927057, -0.06364468299667381, -0.1597260574210778, -0.4153574930677556, -0.013009361810410918, -0.10343875622624399, -0.37297813897263893, -0.6914885743339111, -0.4933916514937875, -0.5050316029091736, 0.03298259967866791, -0.6479194094338325, -0.8017340699937345, -0.7997386373432843, -1.8351599017438402, -1.86890034980294, -1.8405329501237524, -2.7154739421090626, -2.299635678074549, -1.7446742396591135, -2.414306873745882],
[0.20461102726621536, -0.2815258562850416, -0.19228861204980235, -0.35785822759640185, 0.12550050073927593, 0.05832618885143773, 0.22690608313986388, 0.06062300598029569, -0.19287427420828882, -0.21184864147088203, -0.1474256554570372, 0.038877486076584186, -0.0519404293497886, 0.08860217668187424, -0.1401648813135771, -0.2426500323812793, -0.10646820160430713, -0.3952341621275716, -0.06490106307601926, -0.22767867912518874, -0.04927836450542389, -0.14729942454482195, -0.15209630475516311, -0.16952253962604708, -0.351348605850567, -0.07057470772081295, -0.22931605298232108, -0.28483572774728944, -0.6271905351661514, -0.5907803459154157, -0.27197971649615227, -0.43671252886174344, -0.6350051913330576, -0.7499791166427804, -0.8012161181917657, -2.3338843432445864, -2.029559957175166, -2.14229366203902, -2.3239365762415956, -2.3292952377456113, -2.0289462903019984, -2.682202225715142],
[0.13260149069659113, -0.19532894857942462, 0.3326409759064575, -0.344091779912579, -0.3043610096267086, -0.09716278965605168, 0.44987631937423656, -0.20217613665565815, -0.37009738686070326, 0.24080024333352779, -0.5152261955484762, -0.39204775037283934, -0.23196632160415148, 0.16081613750980364, -0.4450436533170348, -0.5061394288113177, 0.7514513597775654, -1.0094942963556555, -0.35692382388048705, -0.41643633607596253, -0.07536732970798471, -0.6090221961010663, -0.3461174671169418, 0.22757383672189085, -0.25164923235064895, -0.3397621002505583, -0.18323774436207063, -0.08422754983392498, -0.8751409613780331, -0.6268577450080329, -0.6650481308031592, 0.18440146361221016, -0.4743029282172724, -0.8005252227837467, -0.6793272397052468, -2.1532809819915606, -1.8368952673128323, -0.799764343662592, -3.2352644201121112, -1.484267340619942, -1.7640717982496807, -2.0535918151073362],
[0.016135750034300053, -0.22811196326214733, 0.19989054111542964, 0.40763043675304766, -0.0091082522299277, -0.22066015929769583, -0.039344901694600594, -0.06321078242261598, -0.3135030454257919, 0.12438731092111724, 0.11580023878318195, -0.11547478787849616, -0.10850890771832405, -0.18213379853616657, -0.09166838688511672, -0.04055032084802321, -0.2068642875498535, 0.08873111466990653, -0.20956488277226643, -0.13415897281994119, -0.16735968031966225, -0.2637771401026508, -0.4371125674852887, -0.1694091681800447, -0.20258101266887568, -0.20163746913033473, -0.38987840139887503, -0.16701255236124132, -0.41335891102692973, -0.11682084075119746, -0.30844700277584636, -0.5892342186368518, -0.4446176234862163, -0.23923989177824928, -0.4092596465161246, -1.3920637442525339, -1.1304750323340507, -0.9416459433555029, -1.189032430489802, -1.1286760131813929, -1.2279875772730326, -1.3334634091344664],
[8.53756497118109, 0.31223724662668073, 0.10927060749543416, -0.322520916298693, 3.208457159402126, -1.9951667759695597, -2.0867827671910444, 1.673224673769932, 1.1811911804010302, 0.846499388439438, -0.5823951788756955, 0.36746300509352237, -2.423888491697011, -0.9874208652166439, 0.6042859880981329, -2.035521557147888, -0.6173251223328549, -0.5360901043031533, 1.455358763593826, -0.9110503811551123, -0.4175316626649191, -3.268210940091702, 2.449139770920623, -0.30750442196745537, -1.4026732372962398, -1.2218926821589071, -1.839488152047383, 1.0978355010658578, -4.4924477184944545, -1.1417811358620045, -0.8101814860965459, -1.7317635136375729, -1.3147040142270359, 0.28737797612229343, -0.7657949869796411, -5.226781483869608, -2.699529167198122, -1.7875001775316408, -0.13097131714228413, -4.205566878581139, -1.8060944731001742, -2.1185521420308278],
[0.1394091965480264, -0.027877445505100542, -0.11408659625198003, -0.08753307963706533, -0.04826981390618166, 0.010781703077993582, 0.23198981092008097, -0.04982117143993989, -0.2075693202728036, -0.13545162600792826, -0.021574541243869476, -0.19939519953788418, -0.2005458595933264, 0.18520187820933634, -0.10140330619389022, 0.142679572670569, -0.191532909098846, -0.3111577300967517, -0.08186801594422162, -0.0014636413308007315, -0.20191592865168537, -0.24153255999198175, -0.404856976881936, -0.35261005831549486, -0.5262559201351464, -0.5493588483034174, -0.45778149836582166, -0.1875817634523521, -0.24759605212452732, -0.024572562351227033, -0.2581230590542439, -0.6801719398596057, 0.24326908075702322, -0.45865318883155304, -0.5485721910492156, -1.3607728916222253, -0.8003906713108931, -1.4977085919628146, -1.0172348450923698, -1.1933824595489293, -1.2871209725979778, -1.1073797329669328],
[0.11490070256939255, 0.26523692052056475, -0.016784000500582544, -0.2043216327862761, -0.1671443212401082, -0.05091343350464799, 0.0673227876996548, 0.16388692728635054, 0.1994264281352598, 0.22543837096621497, 0.04285887334876507, -0.10241277510762135, -0.11526527006526437, -0.07373606544281686, 0.051399291924203255, -0.26274255569837424, -0.11320656589394043, -0.057602741852505274, -0.2393549393156378, -0.282921135007824, -0.14456980132466657, -0.14027228723043061, -0.42438252458931053, -0.11159705692265441, -0.42568289656295727, -0.2171216706791732, -0.009205660901444245, -0.15685218252195784, -0.7591171671268665, -0.625608998078245, -0.6459949294671543, 0.023847329599996244, -0.43384395734531045, -0.3059859931723499, -0.3253402016942624, -1.8365801143457894, -1.3469999136838458, -1.0156016274193187, -2.4161176979160555, -1.0311122809288817, -0.7810192445553181, -0.9647434639436162],
[0.37192040951543065, -0.16887065670679446, -0.3548669601165717, -0.28526586435206147, 0.11433745914516494, -0.026073833680580968, -0.01864327016871069, 0.06377262476404896, -0.25319929134156377, -0.1578720298887042, -0.12263366808379235, 0.37875802150015236, 0.025011690094114337, 0.08665456256642493, -0.722320231025432, -0.4455905419844645, -0.3290557823923073, -0.22647464200185366, 0.12375085205723442, -0.3037992095929713, -0.08466549017472239, 0.5357207552275199, -0.8257641431388099, -0.1941335194334705, -0.07576912963730172, 0.048970955759443255, -0.4459584664408844, -0.17650854820921374, 1.1716093993970622, -0.7909869576517865, -0.773235522141335, -0.26172333251559504, -0.21684433583305326, -0.6321806790130563, -0.2785949365285234, -0.8913601017648529, -0.1978457238021529, -0.5294945282263527, -0.45964997971099925, -0.7374753797324055, 0.09102952097076066, -0.8890192273515697],
[0.11421660409070594, -0.4177968735097062, 0.1775680516557662, -0.37575470054525706, 0.0928504205450735, -0.13766008208936711, 0.15412807126420744, 0.01800517192064451, -0.368008754694799, -0.1529797514713959, -0.340597649262817, 0.07574386486049767, 0.05092740488550692, -0.09127854034210418, -0.03525402488902963, -0.19110401687912168, -0.10496910299162625, -0.2496731061429917, 0.10759849972964279, -0.2861759839599708, -0.08594285837990918, -0.2277541471156513, 0.11671863460263776, -0.015525275297852111, -0.2926513201850695, -0.0755248457293008, -0.29437023975936194, -0.17290946818801037, -0.5657276653390128, -0.033683398616204535, -0.5678456095088957, -0.11288395430162966, -0.5125060097607463, -0.5839818676740951, -0.5704168683231157, -0.936349114537157, -0.791432745136242, -0.9957387321540837, -2.1971046393478706, -1.1761200811415677, -0.9570733570814441, -1.6414856314631676],
[-0.13044728253695045, -0.179845361487185, -0.018906358341486483, -0.18339008443229687, 0.06233611312905931, -0.040418974486112105, -0.11974755797242, -0.045698328063172575, 0.050981030664981684, -0.046039692096161496, -0.07313048039636409, -0.12500253505643635, 0.18765932941293959, 0.04319492815625757, -0.4180998353604607, -0.26623849537850786, -0.2895729744463201, -0.5320304118162259, -0.1508934641672668, -0.2637045430636755, -0.07975399204495269, 0.31133537560489455, 0.03417335672718179, -0.2165742299661025, 0.11277472303149179, -0.40994547359393513, -0.24417368240427872, -0.07914228606719687, -0.1792460527471092, -0.4271187628279516, -0.4187153313875718, 0.03065141950718031, -0.5625288346972874, -0.3653924682452902, -0.3708077658026496, -0.16831505109236564, -0.7944128067808993, -1.016940602869222, -1.0409993135771232, -0.8116838418754421, -0.45665656126143306, -0.8135758151013113],
[-0.06244956210199553, 0.115422249777993, -0.019044186204078554, -0.18869613804331578, -0.12782209977645315, -0.005541571740132504, 0.11252685962573174, -0.02504625799300111, 0.006851714413665498, 0.03261344274195888, -0.18368456025797703, 0.037830165541884064, -0.213229974056299, -0.17120123132607207, -0.0758665576270131, -0.3290451179696417, -0.11826792092649234, -0.2535947319926329, -0.2793496180383504, -0.12851602261596046, 0.08763098948175302, -0.3532879724578511, -0.5388942532407067, -0.3647109645157114, -0.48613976474623805, -0.2799971647209045, -0.38124033138043956, -0.31677095656774845, -0.6513596452707427, -0.5267013409158797, -0.7255694261413026, -0.2961335372512104, -0.506547504000414, -0.4078932706268466, -0.7665582583282302, -2.1145267329072026, -1.5132475014353768, -1.6947437146100814, -1.8919805142423758, -1.7312160399486665, -1.4625981201816758, -1.6155480979503558],
[-0.08735164576176088, -0.041372982328254565, -0.02349088907638003, -0.22555667720945166, -0.07687030610436346, 0.1223387185458971, -0.0451894334312088, -0.11706814349443384, -0.09827832918256409, -0.2879094564654089, -0.2689699285463982, -0.12522477970701243, 0.12788814049256297, -0.08089896416475365, -7.44588380722133e-05, -0.37569816849463455, -0.08752173337414128, -0.35435228666850077, -0.3102509739429928, -0.31132059401401535, -0.22573539757746988, -0.41777886575891615, 0.4131136644823148, -0.0852492174132569, -0.06364586384219326, -0.24151055611326855, 0.12517498187849666, -0.09166254937843644, -0.5643067932675441, 0.36927151717339657, -0.6698057968063131, -0.5316669197351315, -0.43879704271348596, -0.40484807906791076, -0.48224927361506326, -1.4424752943552477, -0.38319854285393434, -2.089948075958306, -2.4587859736373683, -1.8952700404330765, -0.942422701210864, -1.0563121198242873],
[0.06064878587191199, 0.019628140326614304, 0.025045669288706295, -0.32723388894473127, -0.0886135716690281, -0.15968280798113316, 0.1254402304652517, -0.004994017865971543, 0.12327652553477698, -0.14831545977129615, -0.40282265493893543, -0.20435631849483585, 0.18419088555477955, -0.1362910642767175, -0.20194795997479228, -0.19954138792127443, 0.0008442442846668385, -0.6174362921462578, -0.16153072069495572, -0.44635156141798316, -0.12235038215036381, 0.06985791451357877, -0.21076999721169715, -0.23530534100422176, -0.3265373643756064, -0.22293356616793605, -0.1538552819566689, -0.10646941696980851, -0.3145639853587994, -0.5689308553207415, -0.2578029343095846, 0.23547381580450713, -0.425881309112323, -0.5481532982018593, -0.2590266732575932, -0.6581763714439364, -0.38731623300802087, -0.4708509857396819, -1.8625333009018976, -0.8977455187895916, 0.08740028573900051, -0.999401063740351],
[0.29499114623246503, -0.18403073120798824, 0.5876120523713879, 0.08839539362985578, -0.2431649296339296, -0.12947715072444818, 0.28741565560913335, -0.05031961774274682, -0.03585633814075824, 0.37080429749443156, -0.18860723079962924, -0.3055655320313513, -0.02865194937155661, 0.12656327347310994, -0.033510898338482004, 0.16400261426031165, 0.16708464028566514, -0.3216222884704797, 0.0321019428597406, -0.09118140034270573, -0.052463315084923585, -0.1524213093764529, -0.409713935888375, -0.15593192423783414, -0.325859442136877, -0.17596305485487543, -0.31919779190625325, -0.20520493641161047, -0.6582428922395274, -0.40189116475447106, -0.6343794045276872, -0.35624113629156495, -0.35935347027000075, -0.46261286241747374, -0.37828133430858457, -1.9684246586805227, -1.4242456746969765, -1.6378673911849284, -2.267019562976487, -1.4225740623872858, -1.566629785623548, -1.7020667883971643],
[-0.40030845122103026, 0.18032018284692367, 0.07891125925054839, -0.45231482762210695, 0.05729905251853611, -0.13957846115298328, -0.06712163263391457, -0.43306692124701573, 0.031879941239616114, -0.1949959615248984, -0.07385852964678175, -0.02658133893782976, -0.09093574075857858, -0.13238133563223967, -0.03627710818642891, -0.04421932555565326, 0.04106426120768896, -0.2965264796850494, -0.16313701778607842, -0.44741873462005494, -0.37153178173596474, -0.4485975928240027, -0.25163006514387354, -0.2252180517435106, -0.4882239443511581, -0.4266211014050815, -0.39416784728261234, -0.26948254474426697, -0.9832326915708998, -0.5573085968596109, -0.39109932162779376, 0.0398605573120845, -0.6568692079711729, -0.6161202500837579, -0.4709965665255562, -0.4818003119189236, -1.1595842050625498, -0.6501115590807585, -2.180707995104026, -1.1244689054972392, -1.1145706766037262, -1.0905871219383443],
[-0.07863908308718785, 0.1467682595280958, -0.2169904969238105, -0.0030566361530689856, -0.10309239343199052, -0.08247506340326777, -0.22501405538659994, 0.12002513342843812, -0.1969769489853166, -0.11128037897532647, -0.05759537799654757, -0.13847240491388285, 0.0034111989226422875, -0.01675145712789844, -0.14682500856681632, -0.1898134456003282, -0.2734893801282477, -0.23940058113199897, -0.31025559474268805, -0.18309480726269656, -0.08394941089638419, -0.1512097923238752, -0.30579839388617647, -0.11762229076116956, -0.16644067674776483, -0.1174639689025419, -0.43233268623756754, -0.35655068722609684, -0.5713735560622442, -0.4265500643761307, -0.3985682689057811, -0.4372645407655329, -0.36069575486640876, -0.3941341172427508, -0.6629668160457072, -1.3504371618251507, -1.3212995788222992, -1.4301432071395213, -1.932999494848756, -1.5169093926303876, -1.4522762312313604, -1.5708840683206808],
[-0.07435618172663801, -0.12674003673933465, 0.14563016181534372, -0.11072808536800029, -0.027805058403243064, 0.11841720028718646, 0.01976760587142272, -0.1554014277986272, 0.028465184231813403, 0.09295560933279728, 0.024945158526201418, 0.019989168607489543, -0.08127156769450819, -0.22370543738082274, 0.020486932537209467, -0.16464961946205425, -0.1292372611607077, -0.3435775633926828, -0.1861532635758224, -0.2728909048185864, 0.056545250863025615, -0.029594722852639192, -0.073254860547547, -0.008994544048159787, -0.36357244953411616, -0.0639044181479296, -0.021990979631067553, 0.011372222047451058, -0.3768086817626023, 0.012761548594164472, -0.3356398482011259, -0.14444863652873152, -0.08292960377230311, -0.16270661002182543, 0.08235225022033642, -0.30113325543097896, -0.12359295581406342, -0.19764122778142706, -1.4459946373473151, -0.1290451477635608, -0.263171044615742, -0.1691342503880348],
[0.8652083786379112, 0.3673148993317549, 3.9014550600426285, -0.4191993792590889, 0.39917528025390114, -0.8687104038302704, 0.5544001496167396, 0.9525706574200616, 0.3705576899012056, 1.894699550965792, -0.7295771203170491, -0.7276375223933241, -1.272053799863962, 0.26655766039017986, 1.30202760433772, -0.6792775675050161, 1.213445042705059, -1.0415875910557195, -0.5210265825589523, -0.7902666466018712, 0.24722070380184552, 0.6748837793571644, -1.1790846041163152, 1.9401784492187069, -0.840020880035236, 0.009464270678539449, -0.06800435619619678, 1.0242640103159082, -0.43407562817248224, 0.03525298101337937, 2.289736201093609, -0.21681564764652372, -0.04147747226691274, -0.03949617664830245, -0.007940186173078045, -0.7776541215417847, -0.7742394849649098, 0.2590135915430682, -1.6087618988787604, -0.8519607522268561, -0.15591322558128493, -0.9906305722446888],
[0.05296072394814966, -0.09332074743236443, 0.042757013088352676, -0.1892000300706115, -0.0018042701778092048, -0.11582970424394856, 0.028167569428684834, 0.0363534231461341, 0.02923289162474173, 0.030573227018262077, -0.11885063350460574, -0.060542567480919414, -0.14266476146381277, -0.18850300035822284, -0.11253562844146285, -0.21772958883412213, -0.1092242764375649, -0.41150164903499076, -0.2579538660728547, -0.14643885973501197, -0.0969029769656491, -0.3609063200203593, -0.318423045161813, -0.31239515518493477, -0.5823667612396197, -0.2890913046062791, -0.33343323574585426, -0.03524110003646493, -0.5932510148738857, -0.48358667318397197, -0.6299855621448498, -0.21964675452981167, -0.3131695612540018, -0.498897078489977, -0.44493938669646904, -1.6748270632195865, -1.1635867961021968, -0.962457009873988, -2.2065030173719773, -1.115365724072104, -1.2367332933989366, -1.2522930045283485],
[-0.028686128799443808, -0.26284196635364965, -0.09781449447253661, 0.0072155081302739555, 0.1966039031638762, 0.20471710756375128, -0.42132814727395806, -0.17940399331072732, -0.14046739592627133, -0.06623828515065294, 0.3495002113620386, 0.1444622092619083, 0.3724060255085732, -0.4061169755120127, -0.4933099681512922, -0.5345111842468144, 0.08214571856312132, 0.25210460183506256, 0.16438099966774214, 0.08940581790906164, 0.31192366370661356, 0.06452667805353877, -0.41584941813781723, 0.23517843090710022, 0.7382571339873248, 0.23906444470662658, 0.39846328366451605, 0.18852550204058582, -0.2984573782752299, -0.7191170538649678, 0.4916012020780693, 0.7616035156432279, -0.20768840612214248, 0.15221370362419653, 0.4361535880614217, -0.5444104466334029, -0.275804750339344, -0.17894120203390085, -1.0515060566796777, 0.3406819615823804, 0.2904954031505226, 0.5502261083846308],
[-0.005318671528714494, -0.031333363870930366, 0.20420892897053297, -0.28589105744854043, 0.06928320546906902, -0.10371222817391698, 0.12338868683095137, -0.06501850786701892, -0.09488208219326638, 0.16458954075874016, -0.22941521099928508, -0.06294516545918247, -0.04710468419661355, -0.09741629531932823, -0.06659885195932468, -0.10531338125127358, -0.030905724294924778, -0.01232235650277301, -0.02186355176031011, -0.1805030026848038, -0.14785861361624042, -0.1700037583894991, -0.3715858369421589, -0.07102661183893746, -0.41789327242253976, -0.1553594102667959, -0.2277195182001101, -0.16655536582557154, -0.7241081724616552, -0.49068520234289914, -0.6327897808332439, -0.22714902058234754, -0.5285745222379574, -0.4268018434321789, -0.8052506227363523, -2.923990646134155, -2.4952011681831765, -2.7866693986976228, -3.1081126691533862, -2.8406482961030024, -2.647967451651577, -2.8574210027229245],
[-0.3981717563226416, 0.3685193657676251, -0.480571381967457, -0.22955060384484136, 0.30071257541671287, -0.025075088463409837, 0.16249480566900332, -0.4961960751757056, -0.417478900926747, -0.5930237154215648, 0.6159664754659357, -0.11089682209169666, -0.3033139281018053, -0.05985123069259031, -0.4177834548605532, 0.8757966401198368, -0.0883903604611332, 0.7165200684343538, -0.08699242725039984, -0.1679275566269247, -0.09721886655519361, -0.7212777069466784, 0.4322996743130112, 0.3535535487528332, 0.6192073602562067, -0.46043198917707623, -0.30742881756227897, -0.2749058609575896, -2.448268842399481, -0.30487613003239405, -0.4734628352998046, -1.3913210561248748, -0.6049805388741785, -0.09556573125369547, -0.8195227670746874, 0.6950146162414509, -0.5694949833545493, -0.219980102883224, -1.4120663028190592, -2.2400172191567975, -1.5039581835390914, -1.8338246076724676],
[0.19116286932353968, 0.12619262363490166, -0.10485260224579016, -0.12455987990757945, -0.008101077634711677, -0.19090500964058935, -0.05393140289791774, -0.23969150817923746, 0.09785810828816807, 0.02177527267796902, -0.4708780781722123, -0.3420207547500391, 0.04719783886986438, -0.6210174925316517, 0.12468929326917318, -0.3124744489367378, -0.4636331827001416, -0.49081632348089127, -0.5755150587987035, -0.06641303766113896, 0.07206627305266428, -0.29625837839238495, -0.2372788064600285, -0.35562215696504096, -0.3724817067433306, -0.3903601442180594, 0.2544943369014732, -0.09534362737474336, -0.48493971324771884, -0.4773609673845931, -0.8479935857920491, -0.037502801714295785, -1.036427541743313, -0.5427847673312695, -1.2113959134029046, -2.507437267267968, -1.9698308436363652, -2.373477279059096, -2.483716186554172, -2.4741741601204112, -1.9833681427642442, -1.744069486709486],
[0.1860700904819229, -0.5148798912643733, 0.3240293158208753, -0.05194047107529866, -0.5018462759686895, -0.42677939137339604, 0.011110676655836843, 0.024493375459702982, -0.3851183343395588, -0.025200963249246568, -0.33473989236020785, -0.2638806970664517, -0.5987074716758515, -0.2914748691309961, -0.2411871420674322, 0.179071669312947, 0.22461615295141457, -0.32612298293650693, -0.4090630529620934, -0.18890319950673196, -0.3833680081292266, -0.5063508522779299, -0.8034031443551388, 0.11284015545005603, -0.976755028863473, -0.8158131887632036, -0.9450028418492029, -0.4131148607491483, -0.8110676273940743, -0.2112244811913227, -0.6571494106814075, 0.01136971653331623, -0.5364850435326751, -0.6269763776133794, -0.6073881548379922, -2.9754349026756413, -2.4731990727300732, -2.8039519287645605, -3.2198637350524266, -2.7518879437361514, -2.5017268863221394, -2.8899619453690484],
[-0.11510322339128372, -0.14909832741560053, 0.1582603836878403, -0.18032955355628005, -0.21014181326449805, -0.08959174215112978, 0.02545277762033458, -0.03345172434852728, -0.1769196454269853, 0.21239822115878093, -0.3296793670787053, -0.016107181517595184, -0.24832253812068886, -0.06263738479651863, 0.14536703223883846, -0.010428941200330914, -0.020040843957415958, -0.3656348008633814, -0.2786908647299206, -0.23352642067474597, -0.17660331897280315, -0.32298119666146763, -0.49570579323085007, -0.05729898382713653, -0.6138657050829003, -0.34214819420009623, -0.6223349536393649, -0.17801163790452137, -0.8767190902628903, -0.1788485910831325, -0.4772596593303975, -0.344596845011776, -0.5241541522440413, -0.362068592047019, -0.7402353865075747, -2.4303544442909493, -2.0110325689016717, -2.410631483540231, -2.6158165224718632, -2.415485404984572, -2.231430082846974, -2.2708685920513507],
[0.10111946060233235, 0.07857294107721093, 0.02589041438216929, -0.2919260679239225, -0.21505043910645488, -0.09177676432045775, -0.2960474641195631, 0.10903430999907619, -0.2902427233421892, -0.09188738667564313, -0.33258406505280985, -0.31670687604846354, -0.34540504794382526, -0.3504716488422476, -0.17484598548859717, -0.42461326004499556, -0.38560107178250796, -0.5300546600143111, -0.5484463705749097, -0.37773155463617925, -0.44284402091762376, -0.2523019589947884, -0.5811071796823292, -0.5951101244356415, -0.5294981075642233, -0.04226873484878528, -0.6067983219663253, -0.2699945217192873, -0.5516328802012982, -0.9393942579039529, -0.6610905764059075, -0.19247713525821433, 0.14056429322836933, -0.2587511321762119, -0.6555851365378689, -1.20634535152981, -0.24307750076288778, -0.687786947978666, -1.187774515466517, -0.3393054268125391, -0.3733532334509274, -0.6290623300316757],
[-0.0863031289219767, -0.1296988322101729, -0.14909220622260264, -0.48822576826707437, -0.5019478716457685, 0.13999672069501123, 0.16261715135078497, -0.02615158265328041, -0.1699973829826997, -0.2571884662869588, -0.4508348362030526, -0.3262132503702284, 0.06687275129384576, -0.18156133869530675, 0.10546985652136308, -0.10487302663137635, -0.2301109921213108, -0.526920616516891, -0.5171803796150142, -0.2803690474273158, -0.4082626742217746, -0.3464386387816313, -0.40318035634611943, 0.18607429619369203, -0.7119968290103357, -0.534819238086545, -0.1655558851736911, -0.32956517271047536, -0.531715435818391, -0.7006835421526851, -0.3336742075556806, -0.11967262296567319, -0.49618186580027235, -0.9051503286458932, -0.8000951638762178, -2.439450769375415, -1.9087164926105433, -2.35677861495943, -2.5791879156434097, -2.4967193264982757, -1.8657375869135073, -2.271186565446818],
[0.2808295103830068, 0.07843280036218526, -0.06383654984400904, 0.06672707853745527, -0.23978264212489925, 0.01405256268185088, 0.3070727322403454, -0.017541502155847936, -0.21694004398308583, -0.27418113031933306, 0.26181126788582815, -0.2971220273154484, -0.12969318812542513, 0.1578419235099696, -0.23116475264805386, -0.44159785263274376, -0.13065097232016717, 0.626602230915117, -0.18777360596764653, -0.2791924628455246, -0.04911814970699579, -0.1570396045305885, -0.6170273476422767, -0.21353300063282815, 0.4189423474524112, -0.3759661086654898, -0.5029338161640092, -0.20418345974593893, -0.4213703243179686, -0.5877994451313961, -0.25773087786557675, 0.19245676316200383, 0.4230483078736585, -0.5641754334655253, -0.4402590642857703, -1.5012775210388665, -0.5213808660212977, -0.0987425693753763, -0.1102805844158345, -0.16566247320440797, 0.44450758582587147, -0.9448084521525452],
[-0.10700480794422218, 0.14819839038637764, -0.1863732665114308, -0.3138830498338447, -0.151394487511575, -0.11308806066462125, 0.09534826507037618, -0.16851944617743708, 0.19934191246145422, -0.14369500141659486, -0.33067365564195783, -0.20172462117949197, -0.15189551097129259, -0.12438076951375009, -0.15426963780956332, -0.1397902200055874, -0.38901439615363176, -0.37934975787944236, -0.040775145930597646, 0.1712964783896083, 0.16093638633874757, -0.15575981959728388, -0.2176560027666103, -0.4310619036157701, -0.5760578464818027, 0.1934833109594958, -0.10532770989012379, -0.057612996865551595, -0.1987095326490357, -0.4618649684594639, -0.6197581478095272, -0.2843504694340057, 0.10291541723626983, -0.4201160531733288, -0.02072449964057813, -0.8055190985077494, -0.5858127311913846, -0.6430216505981114, -0.9185026810945454, -0.31887082644989306, -0.8332475892369051, -0.7964055422105026],
[0.19800597230057732, 0.1259897050908087, -0.7979468695534654, 0.006060303962009002, 0.42917276377824976, 0.5840620529386369, 0.5556444266725357, 0.21706346778835717, -0.08473813514654129, -0.8349981270656919, -0.04969709627807905, 0.21656910640680951, 0.2956072751677609, 0.24611866372623856, -0.13861964404564622, -0.337412212251195, -0.33507063480984794, -0.09621519561346553, -0.11918418051844425, 0.11565992290556071, -0.01844171928963633, 0.0987879477681215, -0.35180733994601343, -0.5966829672552637, -0.27946507257370484, -0.1879916505955943, -0.4721033993248887, -0.2520825641775555, 0.45653213984225993, -0.7625193421172434, 0.2569884180406358, -0.47389338129619657, -0.32247456891460563, -0.6763314236565902, -0.6936158857172005, -1.3571566450237509, -1.2178633661538893, -1.5165855623975915, -1.1410170333498573, -1.887231474917109, -1.528635558604665, -2.162937847488545],
[0.20071628314550785, -0.06410619183105828, -0.34011764060743827, -0.2544788246744183, 0.16233204722362413, -0.08863123373097366, 0.16227630439944146, -0.1657498547925421, 0.10150057796293892, -0.3519280341486416, -0.5235201409357416, 0.10436232171990652, -0.42042769543107616, -0.04097806379478923, -0.40426047012205824, -0.5046354130447652, 0.05335191881664649, -0.4360055864053141, 0.3885758057213091, -0.10126743737022414, -0.250651581520277, -0.8434506410455603, -0.6646445608266106, -0.7363784256694043, -1.0555283881711806, 0.566759166854329, -0.8364386850692593, -0.41234026904095405, -0.793122465350485, -0.6773364228921626, -0.18843878215562443, -0.4276151603299408, 0.17930203713783135, -0.5947519255757072, -0.7378724231058951, -1.6413941299453552, -1.500330956795616, -1.872439094460462, -1.610321395818802, -1.4724158508754308, -1.7972463417680276, -1.9779683437596745],
[-0.03533723492508858, -0.13119815324609302, -0.08859301243576012, 0.10151665581489197, -0.0676491176144067, -0.05507318691773872, -0.1012233831370784, 0.06650894748812518, -0.09976017416898515, -0.19444366559695073, -0.0627818255224269, 0.017630931761339112, -0.012802579329073915, -0.1394844804925385, -0.18073334140738626, -0.19888085789474588, -0.21110856210056783, -0.029648077773376986, -0.15327697908088633, -0.11941606893558132, 0.009989224886499544, -0.23929368975625473, -0.3332382642054704, -0.05465691364889476, -0.22194056687396493, -0.15162789547073172, -0.23882352272868232, -0.25117645502752783, -0.4610908124921476, -0.26459358430140273, -0.3266368638622455, -0.3987066145611959, -0.3428681785337549, -0.4469344649654098, -0.5362025162636775, -0.6482185170630649, -0.9693740117357291, -0.8829609399539199, -1.530204260391933, -1.299025501242484, -1.2708099413177523, -1.2735902024889922],
[-0.26024518385265316, -0.4637254166730605, -0.14088465413539966, 0.36533995079665926, 0.958059462060346, 0.32071143192269397, -0.17660628382175703, -0.28701262809848155, -0.2588959155654427, -0.5853276228540842, -0.08270154883331839, 0.3343362385563436, 0.0099159447594476, -0.16888751148254896, -0.24645236403319057, -0.5341599410619859, -0.16696936416359284, -0.32976409883066427, 0.5558883553119692, -0.13349050723277128, 0.14618644451385376, -0.05959832607108988, -0.07568085253736585, -0.07724843197513505, 0.018533581301581403, 0.12320460589056607, -0.03742315754316011, -0.176425851641081, -0.5582708880705417, -0.29949963060594265, -0.5704228768496307, -0.1764798883011881, -0.6741841763924934, -0.5599304861013003, -0.7657497576795976, -2.6262880870170644, -2.422207434374516, -2.634125956327889, -2.8974456806457276, -3.1099802308008675, -2.5991285169427156, -3.032565358067744],
[-0.28851689715626677, 0.1956287071474222, -0.1764265423937579, -0.6504763605730559, 0.15305991466808533, 0.15175239873955962, 0.09111556095735437, -0.17192932294294103, 0.02133849974780785, -0.1693110828200393, -0.33511272790840035, 0.10644368748040163, 0.011396688650194584, -0.01908585586787095, 0.058892058749707904, -0.1727716593979503, -0.07878353750112554, -0.8704831031863236, 0.029165188664784653, -0.41961337909208873, -0.19907824049638978, -0.18626740048144186, 0.15670020697422954, -0.3191617998510219, -0.47069824641157404, 0.14487145870657062, -0.020155265077861383, -0.159285397729435, -0.4459738345336588, -0.4841346210191857, -0.36020698468774665, 0.2177454940329571, -0.4416470282999862, -0.5727988143119114, -0.6825160829450184, -0.9854193150023862, -1.0684687583248995, -1.2058657797617411, -1.6406156331183421, -1.3749368254864227, -1.0497045157321445, -1.8308364184878583],
[1.2393974894560242, 3.518834446394123, 3.8029612729961153, 0.9256753799880064, -7.485885833220319, 4.284470917475509, 4.741227160583766, 1.2432511465224199, 1.7155077137200414, 2.669596654061437, -1.324087956660944, -0.9997575026098519, 1.7779483833632985, 3.310829853532772, 1.2527364910120424, 1.0509580179626854, -0.12512104397999313, -1.7797617420228213, 0.4896539630400418, 1.8847866766493802, -0.34773923816881924, -1.272358975425059, 0.3529563004624269, -0.3089301598833767, -0.7371442024155461, -2.76629207531091, -0.8446684508731664, -1.161557523686972, 0.3671399817100259, 0.8990593296959029, -1.4791552750667478, -0.3145595784409503, 4.847249137572438, -4.177652341744809, -1.3128251271145306, -1.916655002941844, -1.1570120910780615, -3.122622622040341, -4.296571127264785, -2.0747031090517773, -3.380167208005112, 0.31576952337311404],
[0.06227208235218862, 0.04874615679951564, 0.22422806605623882, -0.02686499239870511, 0.254662473012496, -0.33308686219107875, -0.09471548654792206, 0.09637563653060777, 0.15817802463122801, 0.16378816851965558, 0.060567324488193346, 0.04421652061786829, -0.2203383686580011, 0.008333080011324873, -0.13332396442304323, -0.21407725888333637, -0.03259116956158543, -0.18695529040247924, -0.209090829425221, -0.24823590211080868, 0.01493486683190374, -0.11698830072383727, -0.5137338119955749, -0.043882966984844465, -0.2121647626424775, -0.22447949294033356, -0.5603457440707654, -0.45335178704976986, -0.6058925976698524, -0.3986174927903405, -0.4538460614644954, -0.08496584862144253, -0.6046862322597035, -0.20664150545240575, -0.4626662077885925, -1.7647389885414957, -1.394209265350398, -1.699032800810831, -2.106805128317671, -1.471280517672026, -1.2379027706844499, -1.6359916268651935],
[-0.03304676656629049, 1.1708002902691, -0.47307117422429634, 0.17670630138468837, 0.5884259265343768, 0.14021016550253967, -0.20911702832180076, 0.3095398835077875, 0.5953600898378211, -0.5870800130619829, 0.6179191702335705, 0.07802384331239502, -0.0035968970512950207, -0.4032148623406705, 0.05063285144519957, -0.007701051201323674, -0.5359058738125574, 0.18503867968194118, -0.21222676974761154, 0.006505402239377637, 0.14566224537547354, -0.023729478482597852, -0.6257308265347695, -0.2889458393657928, -0.19308125694034572, -0.5264350201662307, 0.05454043432040118, -0.24224331988529424, -0.8106811637933157, -0.9548639253141015, -0.5708200551397494, -0.6472371966298894, -0.3050196990365028, -0.07382727644955459, -0.998617638915904, -2.1956709444003355, -1.9703825265042243, -2.635100279763246, -2.1529911728547915, -2.0865267291473506, -1.8540622572856558, -1.1361473389360222],
[-0.2738604563558652, -0.016916186455486322, -0.20546770306621828, -0.4611526563724922, -0.0579688721047754, -0.09485016465417226, 0.008119131521206331, -0.34986966081358406, -0.1195349501570087, -0.4389480763595015, -0.2672927675716778, 0.1301658266211329, -0.26345603171970455, 0.252877224404043, -0.030779542095665354, -0.37855339464403465, 0.0851000810562137, -0.4632902358894307, -0.23709845156627965, -0.2294700883142728, -0.03256106574004215, -0.3068301742298843, -0.33527667752227114, -0.2746887750161401, -0.631038111380561, 0.18524558884965514, -0.36308969124336704, -0.2455089771070507, -0.4698265879164612, -0.7541798628470767, -0.3542475621683448, -0.1633749038331823, -0.20370980743202893, -0.6610083163265783, -0.4128333199209691, -1.3571761288761728, -0.9196633824495342, -1.3801853739427439, -2.073553188125394, -1.042524654325113, -1.3390757721714708, -1.602830852940107],
[0.33297964621238185, -0.32018034708330695, -0.11439842850308225, -0.2882171397722255, -0.07966992642791196, 0.15008921196121194, -0.0200734552005843, 0.13119535220511264, -0.31173220493747855, -0.2946617689912261, -0.1985834950784237, -0.24620414325577425, -0.025856662569763333, -0.029459212715609407, -0.16109199375682207, -0.43741391179613576, -0.02538715692688404, -0.3243355053250165, -0.10217840569195852, -0.2816285062741253, -0.3414839845482757, -0.4724373842092503, -0.5269751039835985, -0.4295640096503834, -0.19091408634964205, 0.0038198724993131145, -0.19904311783955933, -0.46801794923627305, -0.4407158768608158, -0.6776421021803586, -0.243883832144714, -0.5695984448247011, -0.35061892730534056, -0.3378216143307713, -0.6669904226245521, -1.9574357608796338, -1.0168951728094477, -1.1662021189840155, -1.605551950022832, -1.1380813839337103, -0.6265179426335528, -1.3564551898935804],
[0.05594811813285523, -0.02127526886171073, 0.03525074965982851, 0.1526611952130982, -0.047999408616726566, -0.14331136816489173, 0.23397994861747093, -0.030127055526838854, -0.2613545379303988, 0.03674957583532092, -0.1251862672340588, -0.12733750099652844, -0.27367359787208745, 0.24744582425703837, -0.17851464671922443, 0.17252032198831255, -0.06806507262712667, -0.2187008540987264, -0.1367345702719837, -0.2501910173435984, 0.032412784747538, -0.24631446399332513, -0.24810893574882992, -0.1696343320024336, -0.3652636744130739, -0.11933008149020995, -0.6162529414970316, -0.24001809841123545, -0.6417351537167362, -0.3399459023450131, -0.5588422591314857, -0.5991313118327929, -0.19733627657296968, -0.29185481738602626, -0.5122573100024823, -1.7184829621471671, -1.3777811838966196, -1.908426015725371, -2.0444706125777814, -1.5130563565990938, -1.5970530720148606, -2.071350644617091],
[-0.10381847055654307, 0.2388837268486657, 0.09902946798630774, -0.04555302357620782, -0.04759384834534187, 0.03189680936554056, -0.4044649260557845, 0.05644530465314712, 0.07578021645602397, 0.043325668609721896, 0.11447520743652712, 0.05758962811330872, -0.15714523922724113, -0.2050522998156925, -0.24503889115856348, -0.04046939706795153, -0.15712247638536284, 0.03711122594150079, 0.02211325358745923, 0.013133481792744987, -0.3481619426773105, -0.13781655440528917, -0.17031541997799582, -0.3284216322242034, -0.29378916282344464, -0.1719900080693224, -0.3355601620801479, -0.16607241400852482, -0.7391713992955047, -0.37352198125484254, -0.48190290550870457, -0.5806107073187285, -0.4795287563060278, -0.07752715563637014, -0.5028379390016388, -1.5467590727726626, -1.3843152914591716, -1.757963611142912, -1.9289586963640535, -1.5335672965224363, -1.4004342821442426, -1.2979867649403218],
[-0.18374359513713054, -0.10109566787810902, -0.08195048829530846, -0.08494784603334866, -0.1757289119085066, -0.02822718377342071, -0.00495049243964253, -0.271243666914066, -0.813826938477108, 1.4125850316591142, -0.7041105766501582, -0.16713345027205376, -0.5953635211136671, -0.28312287599334285, 0.020696390384712514, -1.9208808911853996, -1.618792759490609, -0.1736939060211863, -0.41967072522026855, -0.22731106924160444, -0.48487048028050683, -1.0806570924984347, 0.9316984831132369, -0.19190210429582813, -1.6445262193421122, -1.0923495610976826, -0.613534816672421, -1.0072996951942932, -1.0403160135504135, 0.6469553792127333, 1.403991224768764, -0.05444136132829441, -2.7099950160795774, 1.239627871278403, 0.15474702163290388, -2.4036221888376317, 0.65273973080302, -0.626112566155969, -0.4430824513811535, 0.2103256309569489, -2.660732050271663, -1.5761831949211929],
[0.1410747643455052, -0.13246134660171763, -0.06995709666010644, -0.09614002215856204, -0.0023530626613146176, -0.08363447211294817, -0.26792236514531764, 0.003501169833649598, -0.05347651046309142, -0.11187710082023063, -0.0739886502053116, 0.1545277242734535, -0.17941899073728232, -0.22696182982718996, 0.14580349170639295, -0.19308092009627414, -0.018974170757265048, -0.21431559490991361, 0.05318092140393214, -0.1442677289637418, -0.24720814848873876, -0.06957080449286979, -0.5320688762266033, -0.17282760033360856, -0.2117745747674313, -0.2754362135326938, -0.18521085468211934, -0.2580184692209652, -0.6725399400866728, -0.3341715542881886, -0.44555572463097054, -0.3912444530388909, -0.5262164026658759, -0.24206333563402146, -0.54159736746253, -2.154475794678631, -1.615966715365632, -1.395753560081773, -2.1071276013994322, -1.5013244324036976, -1.5265512074727414, -1.387600252239348],
[0.8643640352171398, -0.3600384820211646, 1.2240299462914386, -1.1980569866816184, -0.42092166820747706, 0.7367917696067798, 0.2073928747914458, 0.5321652390438428, 0.006854029596239469, 0.8935363385843991, -1.5445316951269894, -1.6757885596225557, 0.3824609545180921, -0.4836215101287356, 0.3530705414907401, -0.08152681642879082, 1.1040769492116613, -1.743994922036886, -2.6225115533616647, -0.9368590300442373, -0.8923617696058788, 1.2501915576411566, 0.11750343139892863, 0.47306866417884624, -1.6763421573780453, -0.7465915842322222, 1.5363569008387017, 0.7464374082586697, -0.47349140139364454, -0.16151027533002849, 2.3852044426022685, -1.3220941730085327, 1.4690200003543044, -0.9789038684745288, 0.4234304892449655, -0.1599440749687086, -1.268479062784887, 1.5910788580205868, 0.15195556909267025, -1.36235778561697, -2.136477761518854, -1.7816839356766272],
[-0.18439170321990353, -0.9004519554278119, -0.8354496315681269, 0.34219944612925374, -0.38991558269318427, -0.0024605291197461487, 2.3029901744726144, -0.15928715249963507, -0.486062098608312, -0.4121775467089285, 0.0729429092881975, 0.21517638535968825, 0.2906734583613169, 1.6713327445740984, -0.09818307544165909, -0.10715841491097676, -0.09371626766636167, 0.0973281138590391, -0.2144339952113254, 0.18763986214657485, 0.38044016128961716, -0.35449142920337257, -0.5940763221143758, -0.6204153898996579, -0.22610458112376366, 0.04341794510261547, -0.20920996616337267, -0.5597562125597432, -0.5853370316127268, -0.4381261282028894, 0.471637095819228, -0.742470394383044, -0.5499325754226191, -0.7716186335310136, -0.9834869187387554, -2.8229623251035343, -2.6350624814746952, -2.8968478539582407, -2.590042078295891, -3.0579477268571282, -3.0955473777728315, -3.437528119510005],
[0.16530973526724865, 0.07928977416662533, 0.21846341003386432, -0.23962353970138495, -0.11512199309822525, -0.05181521067580348, -0.1595451415109181, 0.1267686508418378, -0.2169790422025638, 0.1940760453826962, -0.2857311330563585, -0.09126205979321068, 0.1276768054018971, -0.25127466483271493, -0.1850995713248127, -0.3480862730851026, -0.10153473280555221, -0.06574471575040239, 0.09168015951000556, -0.1336850956715145, -0.2538324929338795, -0.2354437896231831, -0.24369027556340803, -0.10766567682500566, -0.38354285189638526, -0.3415787081120575, -0.25032326082884093, -0.2152363742752068, -0.8119391646979818, -0.5800453869149634, -0.6527301314721135, -0.18280402291420864, -0.5127880318872245, -0.4497905726162449, -0.5794471248644978, -1.975719979359944, -1.5599519391957912, -1.631448608788614, -2.502057163745786, -1.7924510621314627, -1.538239221265328, -1.4475404421340345],
[0.2614386603596654, 0.36838658175885897, 0.07905458238668402, 0.2497844753980247, -0.8370496050579282, 0.28035859510533084, 0.17627162706696892, 0.23075573151463638, 0.06511185859062107, 0.2949827170192809, -0.02414228104749805, -0.5105956843762832, 0.1913969568853513, 0.2250273557355157, 0.337007749344338, -0.09943656293252409, -0.056779865464901304, -0.13382358634930955, -0.8931107326146647, 0.10426205399550137, -0.19981691316332434, -0.3652266809164463, -0.19101804313726656, 0.05533077228592653, -0.3649833092763628, -1.089776614327277, -0.5209503320930318, -0.1791756753593894, -0.5447604270688269, -0.4924396888872646, -0.7583810199038539, 0.09104381355008283, 0.09958148186406895, -1.0071966481145838, -0.6966188744041291, -2.549103829600905, -2.032837663830911, -2.46917648942092, -2.4903183737746986, -2.5128439704684125, -2.194244076854122, -2.036699755722562],
[0.06863423075178078, 0.2860035234324629, 0.020167552918377872, 0.11380955674829689, 0.05902005552698022, 0.19692804164533317, -0.27085406641753623, -0.1705407702990997, -0.07964403921457427, -0.3763002702079475, 0.033669701849428885, -0.028567097766436008, 0.0005572172397181616, -0.03171546853392701, -0.1835949345534627, 0.06956155027827174, -0.22670924093207173, 0.12622318662298226, -0.12038008825845835, 0.06669708013640924, -0.14634574762846184, -0.3145113113633679, -0.3007226393896297, -0.5472295061083459, -0.19586117733675926, -0.533083418006486, -0.49692341670740225, -0.004837956104732551, -0.6871511434430957, -0.5243874562309874, -0.5330734452094327, -0.7825359734010277, -0.3618690118495479, -0.34699794694752517, -0.6259639507427599, -1.47800348257917, -1.8736420125970281, -2.1513947953593893, -1.6272941985945257, -2.1477522466135044, -1.940679623198617, -1.7651455778264717],
[-0.11640222677453227, 0.15802122794759754, 0.13739204520592666, 0.07361115791044509, 0.04031604950789751, -0.2033200894756315, -0.08865045405705248, -0.2482092497629083, 0.09919340008832851, -0.2555598405637318, -0.16181411468774792, -0.2927412074185025, 0.001658538116390468, -0.17347899766307515, -0.22407399426533778, -0.011595895609964747, -0.2823990475667897, -0.3816393595163066, -0.04569089047330129, -0.2020557650551103, -0.21673229607036565, -0.45320299243558226, -0.2415798232330097, -0.22725948530675175, -0.3993854163889587, -0.3434127557984206, -0.23706127624176526, -0.2685276184214343, -0.764138878451488, -0.31507685823316217, -0.46318825850696627, -0.23637672735869156, -0.4562109467105129, -0.19234156981210895, -0.6579171757064981, -1.4653082800417998, -0.9640477672377478, -1.162783821218712, -1.3011865122712654, -1.3351629468642408, -0.8466807385105486, -1.255735780719928],
[-0.019948796928192195, -0.14746936867635277, 0.028307979650050753, -0.11137168492186542, -0.061702343055717736, -0.0665905430874241, 0.011912603622358197, 0.0351047534821058, -0.26001465431047655, -0.06119591146520016, -0.3985565505043243, -0.05724394436187046, 0.22849209432339984, -0.13956800670299968, -0.021912461103112495, -0.23718973208044225, -0.16149377713034613, -0.5424764385501627, -0.27965572010397427, -0.40805596900718516, -0.15962690938800003, -0.3357196484004539, -0.4695667162039442, -0.00920955392023498, -0.5203793075337598, -0.13252595437848633, -0.11028358899712382, -0.24704457137775843, -0.4919863681460768, -0.6958028594002452, -0.472976370523722, -0.2150635534257665, -0.7152628805711118, -0.4815094334975825, -0.640110678222925, -1.6627637857386905, -1.2947604153486678, -1.210739786284861, -1.7519046691789373, -1.4880100153278548, -0.8876140827861495, -1.8982269129554523],
[0.754537085821547, 0.03575857200492936, -0.4539543839971823, -0.2548307410854819, 0.7147471451383164, 0.10565245543787764, -0.3755075773214604, 0.4580578584059736, 0.15997132815998738, -0.4180194393325159, -0.22422762860178025, 0.4962235573687877, -0.10051012430200842, -0.3283578546024078, 0.1761422530128764, -0.037939423070345736, -0.041154939274133794, -0.2766790847160604, 0.16444650569668354, -0.11525288624578128, -0.37652611902899896, -0.42686033208469826, -0.4869272201225364, -0.380825473830869, -0.39998585273528175, -0.23656590489675738, 0.08580404623646436, -0.23838048484486185, -0.6588910194540452, -0.47075149550824374, -0.21970578823594764, -0.0971936022726345, -0.6874565864852782, -0.25695145125579133, -0.6505417397676644, -2.7494094172405052, -2.3680911613662894, -2.134990065129185, -2.654856404390756, -2.709160927931662, -2.059342811713367, -2.6159624083556006],
[0.4515215006100923, 1.5502592079685904, 0.40841711484878374, -0.5114690360562159, 0.997339440318008, -1.5474396958677297, 0.7539681343236246, -0.17700475729028836, 0.7514012816133689, -0.29176293945203163, -0.852266175395108, 0.6105257978923915, -1.0787811006980006, -0.017231400691844583, -0.09059854454095362, 0.28528793634122, 0.07521626709868087, -0.974984790077636, -0.08924513694166415, -0.4228323710820028, 0.15773708628698166, 0.24116712356774742, -0.2070339089800332, -0.8781484880996309, -0.646226244550201, -0.36173340634710843, -0.25034887944191586, -0.33901510885325414, -0.8683417614689304, -0.8674186529839573, -0.6563755572770433, -0.3840479782861537, -0.6530717787510419, -0.12456228185716477, -0.8481586495201293, -3.5676997696820036, -3.26212378604153, -3.7859569952659093, -3.4593342799854803, -3.6931426520000294, -3.415666225687162, -3.7400044491438207],
[-0.05580296474070947, 0.0023441844597687817, -0.16701358656441392, -0.26805012581935245, 0.026291313868409674, -0.10072643361534223, 0.05719624513217373, -0.19065825168476355, 0.053415570784507795, -0.01838393054006524, -0.1408044683679972, -0.12041131224896623, -0.15140512087619784, -0.01485426954327447, -0.024817372025891262, -0.2695741776751163, -0.03838838605420761, -0.43234369623229807, -0.25663942722731403, -0.26526355711147714, -0.25279437208888167, -0.29273873796426303, -0.18702674726113283, -0.44009455249908547, -0.6422614728275834, 0.01381256643102657, -0.17282525438747506, -0.06750157021310453, -0.49595415386390973, -0.6651884212117948, -0.5409985464682537, -0.12702461162876535, -0.33988516971081706, -0.4477896491797086, -0.6114881814804632, -1.6008473874709634, -1.094519624245888, -0.9031979222659706, -2.073183165955126, -1.170850300655912, -0.9909327532222417, -1.486876897186254],
[0.11704620235452247, 1.3376540292588062, 0.6957148566511331, 1.041043342679283, -0.2625451841168329, 1.7812713048483184, 1.2756330723033362, 0.2171341574403384, 0.20566070191665, -0.16023232779005692, -0.4317532311648718, -0.4552447970591592, 1.4260624127815262, 0.25605682435265686, -0.5126511636112558, -0.5307527259003847, -0.4303889270849697, -0.9704486627218253, -0.49687473585378616, 0.21102310537201552, -0.2260847250408605, -0.32541530428878984, -1.0288949244022216, -1.0943623639883064, -0.8480146510946489, -0.36447667540148154, 0.34215651425414806, -0.4740071822963333, -0.6754829342804779, -0.930542942101383, -0.6483412333810801, -0.8285953643867094, -0.9775443724238558, -1.0181083247572495, -0.8692441521502365, -4.743998515599902, -4.273591877667759, -4.644737216853793, -4.547869324300984, -4.668487042925176, -4.390431763235343, -4.671113996893115],
[-0.01654418114793864, -0.1171677331706377, 0.06653058089837091, 0.03788700281367767, -0.08358430916647944, 0.01302557121752726, 0.08483090281443041, 0.020999318475503802, -0.4283589742513071, -0.09106022990820381, -0.2568885223170334, -0.19045949621474215, -0.4852268180065971, -0.17272587051116886, -0.25407768188390895, -0.5206242846819846, -0.4472968486891152, -0.11271644139488425, -0.47560898575814176, -0.21398667865280488, -0.28953770362691594, 0.06141907615873489, -1.191023292115089, -0.4009353078425254, -0.36317211266719057, -0.3650697121507878, -0.7545058893670519, -0.7309715209623814, -0.4861633759983443, -0.6849423412420473, -0.3609327493992855, -0.5494275042396051, -0.27140718867769736, -0.6375737307139577, -0.8662072925827587, -2.654409172140162, -1.9149038840098533, -2.3136225605736778, -2.3923133800911582, -2.3147172686693263, -2.075591361480875, -2.6171825343044572],
[-0.19427876187157078, -0.023748085664869616, -0.2508527312364344, -0.05699636827338764, -0.1476385811027855, -0.07069258354385415, -0.10044406337273061, -0.04740605656203147, -0.2020154724997904, -0.2760987270392615, -0.18371700017976167, 0.015819524945484022, -0.04326840055026153, -0.09924401015886598, -0.2990440210350213, -0.34338948448862316, -0.210336411417744, -0.05088482209190142, -0.2776538607000616, -0.14331947796986624, -0.11126621455485847, -0.1564671576641698, -0.14289215933346244, -0.46924898129766346, -0.2377182357374987, -0.36120972024498216, -0.1640783144911278, -0.3280971521671031, -0.05385042005452132, -0.6028164989626865, -0.28750884671833354, -0.25311795511226604, -0.2973184671220203, -0.603685180520136, -0.3413658698190187, -0.9987890230807104, -1.102307875188048, -1.012347289768808, -1.1748275179448404, -0.8644658179359341, -0.9832403454864114, -0.8379331028460345],
[0.2643703394061303, -0.5079429003293918, -0.404368373509345, -0.548801041341807, -0.4689610822556658, -0.3057680788721099, 0.5282129510028829, -0.1988024824904608, -0.3369146360515528, -0.6918382384511799, -0.6815217494062422, -0.3443078060302295, -0.2841096274705609, 0.4238332408787889, -0.2741885247325473, -0.7829563575009196, -0.3201244186681932, -1.139188219318722, -0.7180475154713181, -0.8596066239123691, -0.49921079006899505, -0.2600902260664378, -1.168172574721451, -1.142182272173389, -1.4629477196637408, 1.154490773551831, -0.9963790484386504, -0.5603084677186521, 0.12012210720307913, -1.07179880195836, -0.6241705770578092, -1.2549185971961208, 0.3102325766963015, -1.0740077546932694, 0.13743401382407816, -1.0570796043928252, 0.8460359391140201, -0.9737036849879968, -1.564423806486876, -0.720186771061947, 0.3397624124250082, -1.3567636486851398],
[-0.01200800784529336, -0.2152232144429248, -0.33414693708454923, -0.2523085007151755, 0.06418098730443508, -0.03128444417797763, 0.5368910816260305, -0.2352092237829818, -0.025405113894299444, -0.2139585259946935, -0.7305095821593073, 0.41359518109282817, -0.1652934217147233, 0.16078840842760356, -0.3041470299554946, -0.0873266722647505, -0.2039146756266145, -0.5349026688769425, 0.20521013543763506, -0.5480889602543125, -0.40861826492761993, -0.31740776655977093, -0.32302166087664863, -0.5760931099998057, -0.48452130626803386, 0.11673287935213493, -0.4271715163509854, -0.645537268093641, -0.3401703951725322, -0.5467608782032664, 0.04108625119693398, 0.031655498918188654, -0.06140860706581588, -0.7104426902242014, -0.5795334918329768, -0.9478887415853329, -0.8441935812861558, -0.7161744552121977, -1.649201112363076, -0.5130825474450484, -0.9969601741198341, -1.7660444797315584],
[0.009461861524939458, 0.18356910102253934, 0.04127547127030159, -0.09272169838375885, -0.08805991552564096, -0.14017465014508237, 0.1242758007767591, 0.0005769739930986231, 0.1402305161274172, 0.0033937127555618853, -0.113612604105515, -0.14208059285135338, -0.17053462065434324, 0.17852492860128097, 0.06285531756516942, -0.10404747645679148, -0.2729020598746283, -0.2390906213691008, -0.13462551065686276, -0.22114385843098883, -0.3127759033881917, -0.13933915961842738, -0.03580481151773531, -0.43708957840149043, -0.2700168551601558, -0.3536506641050262, -0.5820034835564503, 0.08038951043006334, -0.6499863753582831, -0.5939462801276745, -0.5625080218293523, -0.4262062323441179, -0.05389379829417989, -0.490825040831238, -0.3005617260836047, -1.6666940496420213, -1.1762567920153928, -1.5012703831984064, -1.6610604285655453, -1.0841956600484512, -1.1415272832434822, -1.0810084163160296],
[-0.17539504560631214, -0.16178870837153939, 0.13044121794393096, -0.10759143367139933, 0.1498656523498031, 0.11232116341472659, -0.029198758588243223, -0.10290911914977056, -0.37558551571000254, -0.1526819434327089, -0.03123606768001994, -0.10635804476730616, -0.06673909537890997, 0.07390802909749893, -0.07283134020598654, -0.12249015559007456, -0.4196392696340612, -0.4034441559204783, -0.11228154002116406, -0.32948121152906384, -0.059839696774969135, -0.21268892202412276, -0.20024145561402518, -0.3048665066562511, -0.39001922719653026, -0.1034921870761446, -0.4560584833239306, -0.23766727377924293, -0.8289909447416969, -0.2512214312200786, -0.8133688587784966, -0.2833183770174306, -0.5487825513221813, -0.6800037628039787, -0.6946714762056181, -2.4960940167106775, -2.1801571298251727, -2.653511819987946, -2.601002858771288, -2.547000466185602, -2.365576207799584, -2.5633819275074274],
[0.41823087756678107, -0.1976290031033455, 0.03857323249343378, -0.624729071120039, -0.09619312543261321, -0.16043697379160798, -0.09175893723952447, 0.08675726771685162, -0.3267563073641289, 0.023676988966063062, -0.9211463036162745, 0.07492746716071651, -0.16922894632614194, 0.09646984585297451, 0.2858847478539392, -0.3470164079377208, -0.18568679360384896, -0.9429004030622894, 0.5273565642055352, -0.4997035772104906, -0.34271460922280406, -0.19062342741059687, 0.11260908449722282, -0.3621344329862618, -0.7466457732716434, -0.09984675729791276, -0.7135584453312073, -0.5901051437287298, -0.26677568607691293, -0.8929474108979655, -0.7476990502770704, 0.15967992517889973, -0.3800058696886834, -0.8782421879408384, -0.19644579983122146, -1.4367393482540307, -0.11667797453644409, -1.3015744938418279, -2.508053109327832, -1.0380771076184796, -1.0471263553712138, -0.6717511557733956],
[-0.18076976203296893, 0.04571464750404421, 0.11889864698304721, -0.011583716115670812, -0.018411443848930745, 0.04847806648009771, 0.006476788855079667, 0.041475738446536674, -0.18159635257696194, -0.10024308132082498, -0.05432632099143348, -0.04171526329294319, 0.07287627718834788, -0.17013415083391326, -0.2649100308540207, -0.14969719921664495, -0.09138247954850952, -0.5178485840049654, -0.2652721152501841, -0.24102729194201902, -0.11599699408064014, -0.2394252246864263, -0.2680764831652725, -0.16895206352318226, -0.49657586319602015, -0.26265221641797337, -0.22471213017487185, -0.08373596136471156, -0.6193493121624505, -0.549729243251472, -0.6068630714457092, -0.10667849563357513, -0.35637871622848744, -0.6079256802647489, -0.4140336817581139, -1.5096725963593176, -1.22197604964741, -0.9889382343560917, -2.226148867425783, -1.393936045177124, -1.3546019509483918, -1.655874929256116],
[0.44279036873603234, -0.17597539073836074, 0.5680852588452937, 0.16271710367583686, -0.06751913188535451, -0.2230874912084644, 0.012926837693717367, 0.328190343012126, -0.20010583790551964, -0.11485343279772164, -0.15953319050334352, -0.33076723987860207, -0.27977128126635914, -0.03808940078082525, 0.04621709738242138, -0.28076836005315453, 0.27114895486274626, -0.08337505017142997, -0.34374517024085693, -0.3065472334059543, -0.3018173803511416, -0.7269642031061924, -0.35466624145779935, 0.008065225768284454, 0.04578782052559293, -0.7540965448427018, -0.626937283312445, 0.27409450386068035, -0.9450623262466081, -0.31003369804908887, -0.7004262210733851, -0.6685435723839191, 0.13450845812920262, -0.2948644494957903, -0.4526082953688001, -2.608224787405519, -1.6282182475155245, -0.10597211373904243, -1.4040175443868823, -0.8901901909994373, -1.0846182025326903, -0.13968219990330544],
[-0.0767858615223968, -0.40655710730764794, 0.3410698766330451, -0.0915388173533823, 0.13661359965343695, 0.1207696099647256, 0.09846470064379649, -0.07812908088452758, -0.03471332524749899, 0.19856716343539152, -0.5109938876776542, 0.31956668436710817, -0.12315417340368323, -0.2064412127292508, -0.010163566162692829, 0.1181563192917537, -0.376734406521215, -0.35219908133398165, 0.27509444393893934, -0.6861749375609221, -0.3266281003566603, -0.2705831201379347, 0.05158728608226709, -0.09577995721352892, -0.5302512700407968, -0.19585386043251127, -0.7868071290509654, -0.4458760145566235, -1.1358615658718691, 0.3755708645788356, -0.660141412192327, -0.5607151687934722, -0.4212885865615454, -0.08753320228273896, -0.43644699347853494, -1.7128625099554062, -0.4819388322757224, -1.866009345180946, -1.9699218621165941, -1.4400155631394596, -1.6774948969086108, -2.0604120277949343],
[0.21507526735655258, -1.184854812595136, 0.042952845895540374, 0.28186519841582336, 0.1900081047974185, 0.4245770725033591, 0.3434012034267897, 0.08921573160722941, -0.723609581843548, -0.05270626279464075, -0.18935568650495185, 0.18613516433020033, 0.1431729245914766, 0.5820314033568934, 0.18827949118719028, -0.5183879386736048, -0.06412581162548395, -0.09044201984733873, -0.1739256577512984, -0.2731811115764681, -0.17183021722031408, -0.2079306887372419, -1.0319586253918036, 0.21740000580857627, 0.1276737383832922, -0.474477589474134, -0.23052366533853616, -0.33953453704885017, -0.6483119586666338, 0.30921036830119075, -0.15461534364494803, -0.6012793284206717, -0.5882633155509644, -0.7646081587928043, -0.7788858588075118, -2.1533296164595845, -1.718344651574607, -0.5317919203467151, -1.9108547625487693, -2.224334437383838, -2.134681083038513, -2.3328050674962344],
[-0.20745290106243056, -0.08962047897106007, -0.3280066030142327, -0.40503712056391517, 0.029378170295421786, 0.12326977430477612, 0.12137756986971493, -0.023953485629098055, -0.0012538037662745466, -0.25564148074213844, -0.3959052301046187, -0.14876262154831507, -0.07983180539491273, 0.052555742672019824, -0.04884061834758368, -0.16574108683581643, -0.2755908175709568, -0.3514161814580704, 0.02447043649814511, -0.1116901536199443, -0.1443681947744168, -0.1644344381482862, -0.1317747558428372, -0.14112199891416824, -0.4041884375562889, 0.0023918273350149147, -0.17633676602951234, -0.2566880189047685, -0.604091760374351, -0.6845407586071158, -0.3982208338432638, -0.02278993773091587, -0.5362728979697412, -0.6776653964489542, -0.6838985636179749, -1.9123281826416108, -1.718673738948892, -1.8368978395396904, -2.416681796052729, -2.0403819804347676, -1.7499369587648619, -2.390558586408225],
[-0.5619300469278289, 0.12975986215289564, -0.13246840453698283, 0.10650479141460692, 0.6768408310669822, 0.2746186150918699, -0.45629235289003783, -0.5443544529894259, 0.09372966877961607, -0.14643188655903722, 0.24788503125097422, 0.21127368671520236, 0.0993324813131145, -0.11334863822917021, -0.3886229209054493, 0.010089321696454333, -0.16787192376039584, 0.13010560954783112, 0.16231278023223744, 0.10732037219336661, -0.11009377722765239, -0.09396217766236502, -0.05870399945687266, -0.15104524846475845, -0.0679000443343152, -0.20125654546039173, -0.25814345717004283, -0.11266840981919948, -0.7511692542089135, -0.247523505260417, -0.5022481192637396, -0.6426599183380247, -0.6210538901661289, -0.4665103973524153, -0.8419074740114588, -2.253829871847512, -2.230035260738862, -2.8260536466223836, -2.6714351593173005, -3.0540704016498497, -2.621456471055456, -2.812407061214495],
[0.010418280023001563, -0.17605746509332895, 0.05614036316002523, 0.3240766275914119, -0.12411022733588786, 0.0684389309788986, -0.0989231491500776, -0.04771428793398443, -0.08449184034392732, -0.38906012784228994, 0.42951932314904545, -0.17312653550340368, -0.17756071539125556, -0.09505764935951906, -0.2157308953087896, -0.19348025753973722, -0.42301036883301296, 0.26919927633565427, -0.5967747816708746, -0.1316292793181608, -0.23863325023078177, -0.40514416558616173, -0.4098909252637111, -0.670690471553408, -0.14314349732573273, -0.5718026106270719, -0.2493134361398181, -0.2730051955799191, -0.748386208901656, -0.41281765387900576, -0.1523804625099272, -0.5685726982878078, -0.8067366164616047, -0.35084109907443034, -0.6787976199580066, -1.5146794970607038, -1.3895680315475252, -1.6434791497710777, -1.277521103997095, -1.9664890868885132, -1.8288824386537088, -1.8826204147615664],
[-0.19670840361958417, 1.0868996340995754, 0.21142273759245306, 4.343891685746127, -0.8689972441908663, -0.8538419172397238, -1.451632239382606, 0.06516707054490427, -0.1186683915610422, 0.2649378475097755, 1.353378158968569, -0.6592513057899573, -0.48475677973764647, -1.3124584286721417, -0.6480972887746862, -0.4034703142308669, -0.2286969536586446, 0.298363543835193, -0.8415815591637658, -0.49718894581391404, -0.4249508211992324, 0.11095068182041339, 0.28429255169463713, -0.3854944940843711, -0.4969627262579908, -0.3381837705676854, -0.2409055962703795, 0.7659389931829533, -0.5572533669696885, -0.7799405442274757, -0.6690792142413811, -1.1705477101978106, -0.6105231409820651, 1.1908608951482678, -0.45130419159495183, -2.4628344581412493, -2.2882655070845597, -2.6659704872514194, -3.0984531999129525, -2.554395959043895, -2.0468788932458173, -2.5639726802234453],
[-0.04640096686644378, 0.15124107283503826, -0.141517653663983, -0.1278680257366609, 0.01383971462653602, -0.14497904769026365, -0.15371364951718594, 0.053607181942649317, 0.0674640978621816, 0.032182033374134174, 0.019352686523249424, 0.0046195038978430565, -0.07082653812425724, -0.023836589639319884, -0.2322374441754832, -0.2398290785365537, -0.10132970796104869, -0.3085385848586357, -0.031676797356563854, -0.007281948487580794, -0.08725071825886135, -0.26155076046707126, 0.17999354506683587, -0.2641329188136393, -0.3315973073987199, -0.3611232327676246, -0.2195098009224895, -0.050522535500135864, -0.676000030138218, 0.11040582689629559, -0.6275945405917348, -0.4708120327042389, -0.3799609865472721, -0.33809720076497274, -0.39719752193739033, -0.8105611409544953, -0.5555771378991489, -1.1492249646572832, -1.726612068464936, -0.8478579880272203, -1.0861167888917487, -0.7832038257977287],
[-0.020100521203569154, 0.06522298830029163, 0.026082819678546445, -0.16641780451586385, -0.07704325240802598, 0.09181611499673484, 0.09508531258131389, -0.18238087053408453, -0.033302660821858995, -0.2805640968430065, -0.12766500099844416, 0.03946782546709019, 0.05985551981209083, -0.07036716926180253, -0.131657997263723, -0.4057144147137511, -0.08338444655014013, -0.45280049441288933, -0.20864077670257428, -0.4225076545379176, -0.023785621680005843, -0.22203666334970293, 0.02419228387343183, -0.2894859075828592, -0.4666391927321483, -0.014657285786599165, -0.08791241817764342, -0.11551900117163642, -0.5971615603984471, -0.43477139264071746, -0.5483684927194761, -0.33802336271907185, -0.5153594010107179, -0.5051301840851746, -0.5591427981196968, -1.586203885596358, -1.264200189062176, -1.4426266946444528, -2.323732171203307, -1.4482031025695639, -1.3135587495911414, -1.8391738748964306],
[-0.06619223595797259, -0.11864601981064955, -0.001982977174183431, -0.037726253452489596, -0.10277669758081245, 0.11863148285993534, -0.05428321473953142, -0.058577252493136965, 0.03694265116901208, -0.10786584805809711, -0.32723373282054247, -0.17712014375188057, -0.19337817934884186, 0.0765542265857547, -0.313467723219666, -0.23517892451003067, -0.38159520101203176, -0.3565624780807783, -0.45748746287878067, -0.19990575499079208, -0.452392325449112, -0.047936919702304195, -0.2482441112643618, -0.06959546135055117, -0.4038511535060806, -0.8939358086715569, -0.2879283730073034, 0.2057439279105345, -0.5264761052291467, -0.6199068295215067, -0.608283081840075, -0.07592893684939654, -0.14803430731208345, -0.7561695445551672, -0.44895510781149606, -1.558695773941752, -1.1736554832857842, -0.678904837967798, -1.3836319867692486, -0.9046556890714819, -0.9188368689793018, -0.5730254363565507],
[-0.12140502902707767, -0.6216725287238675, -0.05175541847491755, -0.5971621772478776, 0.7003094975235985, 0.09060235094402622, -0.17535596579546073, 0.35408570712483656, -0.5252867760388604, -0.7829281429397874, -1.8489968166353787, 1.538964400464322, -0.03185721278783315, -0.453535096282889, 0.3713974992141875, -1.1123002453786077, 0.006619161333986665, -1.917809775909891, 2.6826727284391727, -0.8483520789634069, 0.96441768162547, -0.6341755693369714, -1.0365729724241253, 0.4296893209186724, 0.5769057361218617, 1.4646555940708774, -0.31089169660617033, 0.08811583833714377, -1.3075407854977041, 1.0010613447264993, 0.3687108519506648, -1.211923259168012, 1.7358304683814734, 0.9010505379400805, 0.8223812512111348, -0.4707606541307677, 0.6617069904577741, -1.4464249092155774, -2.5129324623805425, 0.23794300308774588, -1.8462769140248807, -2.035474430187024],
[-0.7251824177092564, -0.08264786563546234, 0.4974297739247069, 0.9267263694972364, 1.1072957480904462, 0.6313174237212524, 0.24410006955765665, -0.5344633728796602, 0.004968468369936838, 0.2142822624078879, 0.6715452981354044, 0.6180074962402838, 0.28400371796557583, 0.2750539504662562, -0.15999372233052406, -0.03999944907149009, 0.8123779915827486, 0.3538338009202325, 0.8110874165523376, 0.16481309522071394, 0.2473471744968725, 0.08051135198563165, -0.25012396762219763, 0.6260723290770431, -0.06930695138531454, 0.4132513891969025, -0.04320463219178396, -0.09086676616222947, 0.20752575076599605, 0.28367853696598727, 0.8931608336719359, -0.5272971454480407, -0.35378936664098093, -0.5244094386795964, -0.9951734732529356, 0.39781669227187766, -0.5614142811235745, 0.24279483998770385, -1.6777891294708813, -1.772425260645135, -1.4759211274525421, -2.060563212394957],
[-0.17383744032551066, -0.055053288949971566, -0.05009128702438209, 0.019977802775832353, -0.10055346589727848, 0.10825895075100853, 0.012008506950441054, -0.1661720361617693, -0.07684938878866435, -0.17984654249012716, -0.21005052152144932, -0.2783158791130075, -0.0843503836386228, 0.069465434673608, 0.4238301841419186, 0.06416124594909428, -0.07530323627544501, 0.10959662019652454, -0.07846900730261813, -0.17072189428454232, -0.18749579290711058, -0.13317227716153457, 0.36984689668209125, -0.05498444038423153, -0.2821169767824223, -0.3314999938717252, -0.29247920253982224, -0.2955997295650394, -0.3701123510485503, 0.3378953359361987, -0.27717076437544363, -0.7827166213055469, -0.0760658123592118, -0.32907942494978987, -0.357885516195024, -0.03870593537857621, -0.24135173070075988, -0.6406481001512081, -0.9229626525071527, -1.1164834340625382, -0.9670324822501434, -1.1771598859383798],
[-0.0777289747156051, -0.13274639877491354, -0.07024780415765945, -0.620796946050583, -0.4423838688798961, -0.2106513555091007, 0.29036119585428216, -0.39202465199978154, -0.04715126378331681, -0.3238291077441013, -0.7415241184517588, -0.46275292146515185, -0.5224178776744196, 0.12132059563900961, -0.09786275242382114, -0.6196243993743081, 0.06152563561796425, -0.689593228806721, 0.21795357527419523, -0.4889128931133545, -0.14158258803478435, -0.4188255071050422, -0.6038363213630861, -0.37532232530260073, -0.7894738410032791, -0.6428798261665226, -0.7524950687685396, -0.5339626351373231, -0.8808149994838911, -0.6402640432456398, -0.44747972316436185, -0.4445993341117049, -0.7616761260448213, -0.5163143713231053, -0.8752033376096328, -3.999521503476324, -3.487350143214966, -3.9364990499645693, -3.999826148526718, -3.789069734831761, -3.6973383782583067, -3.916047028983457],
[0.2875417726157399, -1.3072758222595273, 0.2987817651322109, -0.08499789505956182, -0.2345555377648248, 0.6072458709373819, 0.9381384035021049, -0.20442465220592576, -1.2065934155754292, -0.28973745604121653, -0.8085865889402533, 0.27606844334324737, 0.6028347065137722, 1.077802970876772, -0.01295270185599949, -0.9374613542191089, 0.18699452649026846, -0.7064285126751421, -0.07937865524162209, -0.017411899317556392, 0.13253732456253825, 0.45395866191658957, -1.4124186945390753, 0.47869815517179426, -0.7407696326020019, -0.4652144615982841, -0.271068716121784, -0.15750472615738065, -0.5472902938113077, -0.1650238126949481, -0.5090188809715703, 0.1314493770686757, -0.4867552589965867, -1.110348987249625, -1.1173545476677844, -2.550446588247728, -2.097246266013761, -2.4531339509952192, -2.9034670067437336, -2.6436649026667776, -2.405222394601633, -2.7331701499255283]]

who = [[-0.24512367230927865, -0.07966944337038592, -0.23448811787575807, -0.5557971336749705, 0.10951043687769194, -0.1350123472305447, -0.905882848854693, -0.10048931621556573, -0.1326932790588203, -0.17786384552195506, -0.11185865429694865, 0.464931830640499, -0.6163228855277697, -0.0915708261766171, -0.38445304419389426, 0.067538633151492, 0.2453212875925575, -0.2630692432286298, -0.3032176693695146, -0.47114371822031675, -0.3477147069329119, -0.3197966053679088, -0.0726804776984339, -0.08104139082903701, -1.3240500568853346, -0.225304181406649, 0.10354708495535916, -0.01808510870700693, -0.2580080710886682, 0.6659925153052092, -0.36550745254220224, -0.6063172700605078, -0.6756261091678107, 0.40500176673055216, -0.005852710279153861, -0.17303364477007577, -0.004956998860733249, -0.933609027029027, -0.5258959719437304, 1.327263371478684, 0.48229711325938673, 0.12012544066529526, -0.42342870081589257, -0.23511774085386639, -0.32203357005477673, -0.15834030599944307, 0.08615800773505393, -0.12773539984484272, -0.3769529650495487, -3.5261654395338042, 0.04740255370227664, -0.30824219464754055, -0.31312892658305097, -0.14045892913170602, -0.3060119757073195, -0.19770579259846852, 0.39718162926762224, -0.43039022267583443, -0.10633644922122144, 0.48146798588950024, 0.002818649926721689, -0.33863103447598414, 0.35001073683474526, -0.23521395876616918, -0.13210171399675974, -0.5689591809517149, -0.20397491460634593, -0.10988053547065779, -0.3922458937607701, -0.14614665864475665, 0.08999236929586071, 0.07101727529940943, -0.8201068575434428, -0.07880949547017929, -1.2219648936795333, -0.14674906346272384, -0.3119963173329359, -0.030735006109464946, -0.11482737203954237, -0.08231486677428272, -0.6137062284112894, 0.06729814344319318, -0.31797187099078567, -0.32318079083013035, 0.017136964545678333, -0.20544578107476172, -0.08555856878343336, 0.016430680871672865, 0.10992144463751237, -0.277828325843946, 0.3642301848695656, -0.039983860786489404, -0.12156130398498158, -0.32576829313816147, -0.003635411955554262, -0.10423695637063866, -0.2439902512670911, 0.003650805411053587, 0.5553008786492737, 0.18479898997693353, -0.21393614787129928, -0.04226967773488103, -1.0787937002173986, -0.018829155725717497, -0.06244360030958229, -0.32798234793770403, 0.14346481176125453, -0.13243470195504792, -0.08890446954856597, -0.4163901994385433, 0.13101629022112063, -0.3141815281720026, 0.025594283439771017, -0.20201420569905507, 0.17329731097936424, -0.1187019977203683, -0.1768501521989029, -0.4590477965940938, 0.02500816584013363, -0.43251483041823574, 0.059700798206239206, 0.1266085722881625, -0.10660282090777848, -0.24147125735684635, -0.08850311443115096, -0.11413029247459372, -0.24476749288008495, -0.056245922355859464, -0.23487716733406175, 0.14558498212171575, -0.036194908155144395, -0.27968093621033785, 0.5454320646028031, 0.09462358708103268, -0.4709589347495047, -0.050839856485318254, -0.03481568009420873, -0.20586232236973973, 0.0842628374815117, -0.11107347097147412, 0.04689290314310198, -0.4154876215836073, 0.6182413042352009, -0.3055057898433698, -0.09343947432052692, -0.147210987739462, -0.5366606647819165, -0.27131728778922704, -0.03950733911351826, -0.1713737885119948, -0.09482454170982219, -0.41757944196708136, -0.11294101944375924, -0.22941872187182677, -0.6354168974744232, 0.10127589129379701, -0.32704400795437666, -0.4120913432163522, -0.12449765736533591, -0.03318142464886916, -0.18535310218276718, -0.04451967617937877, 0.15895568379334019, -0.23285254181279894, -0.15612516881004676, 0.3928689443031507, -0.060030624037013196, 0.05109378088243998, -0.02966472623588054, -0.20563247276934096, -0.18496056192235266, -0.29859158587471385, -0.22398315760773901, -0.058450439752268, 1.1664038185094734, -0.39829291569964703, -0.08917185189857921, -0.06209491228831658, 0.051695368192956614, -0.008614635842866994, -0.06243698748926016, -0.23168964899258862, -0.3988093966285149, -0.08394386632779086, 0.15611157489971392, -0.09041497257774077, -0.31096581880261687, -0.04264609974341399, -0.10616135442806679, -0.13802590822693803, -0.3993379735705011, -0.20653044305874835, -0.3146661803933942, -0.13476474532791133, 0.051074262335849496, -0.08175894166569304, -0.1721198719866375, -0.06031475417901058, -0.05466832021686647, 0.0619613294467763, -0.3476194030565393, -0.16233993031938762, -0.09550048383839911, -0.07619866531276373, -0.35074542487644644, -0.5064720669697022, -0.16575430296910354, -0.05205420123833256, -0.20795186790647222, -0.27529093815763267, -0.13993033956530163, -0.6822479373701249, -0.16953548882051817, -0.21336997132805935, -0.24036561977201118, 0.19462932098301336, 0.22083230013638275, -0.3709884584983961, 0.3407057325993984, -0.19608083484716152, -0.3088087793151628, 0.04866713768165397, -0.7730612037307101, -0.25682111656428336, -0.1916797954667144, 0.3862142790500004, -0.21600345910637556, 0.024548563808095107, -0.16763847281375632, -0.08173688988860865, 0.07956906904349302, -0.10526683745252606, 0.08619422124965655, -0.19256023664975236, -0.04775061271056899, 0.5914166559823931, -0.2895095778056772, 0.04379042164240059, 0.2655059574132143, -0.09246816934520272, 0.23585717873666312, -0.11340991989607759, -0.2791759362666501, -0.11649167388103648, 1.0724837337612112, -0.12591665044064188, -0.14063542306932308, -0.32890393581506866, -0.39488839172256773, -0.21326161230986934, 0.12992650181477267, -0.352646404713768, -0.4043951937145649, -0.05570444480427941, -0.2824948506500673, 0.3182912237468287, -0.1669380950220784, -0.08119432052547519, -0.476761799949351, -0.19819674500268813, -0.013083538432921633, -0.19317278047283978, -0.12635410753294715, -0.22358287238836033, -0.27835524273526374, 0.07507468976465659, 0.0437904403785948, -0.27655359018483217, -0.36267285210687555, -0.5007650888880198, 0.13995422749368094, 0.07219977550662178, -0.1205461066183852, -0.15695742606529722, -0.17927616031449267, 0.04042096300240307, -0.38549133034764554, -0.1900316763183361, -0.009886122538778137, -0.2400440992397755, -0.20326802541043576, -0.24606964679586502, -1.945479384978496, -0.0474446932441722, -0.5444741927870359, 0.03560956144662667, -0.44839182806302996, -0.28002775300009186, -0.021958849312845346, -0.10500643885982845, -0.3160584474598183, -0.19453040712370634, -0.3688542894301877, -0.20297284180811398, -0.05636444970493377, -0.1059691620347073, -0.2973805549949639, -0.11223934498012933, -0.16667730772568132, -0.11447722676817773, -0.2023914792387908, -0.256885205728323, -0.12676574651898492, -0.027135431338795545, 0.039085890152554155, 0.7049898225770576, -0.19520652512971126, -0.23704522861043953, -0.4775020732413938, -0.11099972221554258, -0.7630093664873194, -0.2773577728967235, 0.06983336422055303, -0.7396407115830969, -0.5255774412017991, -0.24841292195357634, 0.09868504489150486, 0.01769787250348701, -0.20140632300829203, -0.20017575520369926, -0.2752572904105127, -0.2327051576893912, -0.1298072119515168, 0.05235598683471833, -0.06970772915725053, 0.049360770350142115, -0.5129999617962545, -0.3044620847856275, -0.5069095969176699, 0.05730970867420175, -0.21327663701126062, -0.18852227127389806, -0.7602529898832565, -0.07852212521093184, 0.1346343059995005, 0.05113211692439105, 0.0973596789339647, -0.5896166369551049, 0.0016475821583996281, -0.08467260877604862, -0.3141715005464562, -0.023173099319700973, -0.22173725881862918, -0.22817580672775153, -0.12395260218365435, 0.14014269382340963, -0.02384256795328078, -0.17056250746881713, 0.15867442882915778, -0.39237644264677757, -0.26456484495893157, -0.20466360758918434, -0.26847488884850107, -0.16100681356309265, 0.21948599117603904, -0.13189256127092303, -0.14629009022047534, -0.40050514558823785, 0.08256964594870982, -0.15586725718581523, 0.0197338263220212, -0.11277595256668697, 0.030403090678653186, -0.06871803565536785, 0.05237977250111674, -0.09807397265961773, -0.27660504447468726, -0.17388116027767736, -0.257222324831362, 0.15842844427764882, 0.31985148800957675, -0.09040164002220527, -1.6350259367241693, -0.2583786347152801, -0.4020536377709814, 0.20333248076938967, -0.12862923656050645, 0.023549535147281645, 0.051954792963185475, -0.5813137050081891, 0.12216200253061832, -0.18543009100025118, -0.03379773938908837, -0.4138219854213353, 0.06775240252528808, -0.17648119927263686, -0.3108701097697097, -0.0864250615385637, -0.2054701582330044, -0.38027522390386126, -0.5530420516736861, 0.059877298304967214, 0.15115223303966957, -0.21406836985497613, -0.25921523932211765, -0.03795546923078046, -0.01643235010273253, -0.5918123198330676, -0.4012234568924841, -0.169730391226328, -0.44632197834637777, -0.26425178556558915, -0.11979611998934915, -0.026435588061876077, -0.1389886198263043, -0.8984032243387395, 0.09377326110992296, -0.5208015991017001, -1.552075310336028, 0.7448514930069449, -0.10770619775996995, 0.05648840555438097, -0.0810295816799912, -0.008731752349904176, -0.15815936892129157, -0.11398540394617136, -0.16492694620404247, -1.1423867723598067, -0.07765786322735983, -0.08501310909445514, -0.19593866218228764, -0.0597660297143565, -0.324526307423092, -0.3450509411043062, -0.2436548013639718, -0.06777636491467316, -0.0046873929059709305, -0.09141196033743564, 0.1892851841258611, -0.15924051059851257, -0.04278262507295437, 0.07590376391144033, -0.06264059577435485, -0.1267203801028146, -0.2810355159098879, -0.3472136260570686, -0.235319603346662, 0.05806192288363917, 0.04872080147662948, 0.006025619904430336, -0.17351769965404587, -0.24154104069113402, 0.030182990325381173, 0.1969077654986041, 0.6148281589605632, 0.6046222216252879, -0.10534636531241501, -0.31296591668833734, -0.17789007537115725, -0.556345697568148, 0.0379770127673639, -0.25360719376727153, -0.18020112805591743, -0.29467306257013237, -0.2701545263662142, -0.17950136005366948, -0.24785000885990943, -0.44507494199319714, -0.29109077367523145, -0.2397467760763314, -0.15555384030223962, -0.2107333257529437, -0.2529887767990814, 0.08321369069644483, 0.011445151171332543, -0.38247920195432167, -0.1515741008367962, -0.22046765832080537, 0.08050095075728693, -0.07991369462468445, -0.04888655838701553, 0.038963546447855546, 0.041597125918955626, -0.5195736018176776, -0.33206944780114633, -0.1021043930139698, -0.10021437865908728, -0.2087265968679597, -0.05366982388386948, -0.3547164313701932, 0.11381586337986445, -0.21058841301666914, -0.2651666455814097, 0.027837486546938982, -0.005842253628299292, -0.5202973980773746, -0.14014358401018578, -0.15127741327848232, -0.03751815766545031, 0.004742190822426671, -0.34013679102286604, -0.30382957701541224, -0.0670053623547227, -0.18386441763308303, 0.5400227632894973, -0.18238141213710646, -0.22427180074124306, -0.31407335604935754, -0.03935496502718429, -0.18665396601038156, -0.13815811361283237, -0.11753938104686218, -0.20673986112925255, -0.1742139868123808, -0.5704024892376749, -0.19286980476998888, -0.37795085016908564, -0.45874691339128026, -0.20247486897711608, -0.09379587693687459, -0.15033860483061795, -0.5616849016300232, 0.1913364977205226, -0.7295126320566828, -0.12119895632192149, 0.05055067856614675, -1.605955832385078, -0.05887714525226752, -0.10146888343175677, 0.023872444879874007, -0.20644133915413015, -0.14276798168794805, 0.5821619332769614, -0.005169602869094979, -0.15492438570853928, -0.07696929305688967, 0.02505744445812451, -0.9649922908124162, -0.15365544594087216, -0.13429408482831884, -0.5036133403655826, 0.08067967233422832, -0.3576874994644435, -0.32474142394163136, -0.06140014517453923, -0.037263651236419844, -0.10025315864456517, -0.005495277265611305, 0.01945428394564271, 0.037582365502254515, -0.23153792086108546, -0.03850449694503234, -0.027712113877249638, -0.23915805358227912, 0.8898110856532563, -0.30235269687036265, -0.2193837869866516, -0.3507901742938311, -0.22601792896957484, -0.30426087977706745, -0.10748022866380698, -0.2543014872930196, -0.23015717607563752, -0.23783046773334537, 0.0923354105342588, -0.09610906570882248, 0.027963872538151996, 0.014817355409393156, -0.15956287688269624, -0.08297350135202188, -0.14031722637606822, -0.1529301489562268, -0.12492593526963569, -0.005036093703755064, -0.5011946934807986, -0.15237998503753838, -0.39870046256253044, -0.14629070777847422, -0.09240270848552591, -0.2918193291247092, 0.057325900491879654, -0.12692838662779685, -0.17389413255777691, -0.2933617986780919, -0.37201503570523303, -0.04594642356002058, -2.0388888808244046, -0.32162910744418494, -0.309342038056556, -0.8528365504890169, -0.25389200385495264, -0.2363420136938466, -0.3733270041690476, -0.02500984343808695, -0.0063659117980612275, -0.35593064071240405, -0.2627217234519414, -0.05290259932542262, -0.06308567849015376, 0.06936722408650646, -0.10763195861071717, -0.0663745668234767, 0.09486190465173959, -0.018597862959092114, -0.16169713057196392, -0.09724442045513645, 0.3878134421572017, -0.03957701955385415, -0.8695493759466918, -0.38077757503601517],
[1.1987404554973233, 0.17748536638432882, -0.31965881776857435, -0.5532748752095, -0.43183338325855175, -0.10716620900122976, -0.7210007235471416, -0.06532881958794516, 0.05348776067143978, 0.007427035348895967, -0.06354689644819732, 0.6084477964053522, -0.82605283750689, -0.2496068459173907, -0.004534955973025128, 0.013164994597169181, 0.13224461332982432, 0.15292988572504046, 0.29892725352043487, 0.1366582491276202, -0.27090935744548533, -0.04984971786763393, 0.13615701935000618, 0.19336797570474226, -1.212431483012769, -0.023787814745549047, 0.1949874099804949, 0.06617119535559163, -0.10978725708117038, -2.2386901635952303, 0.06431530429794895, -0.352307609624854, -0.7581132116351986, -0.3053707548556961, 0.2248264326528119, 0.20548377567759624, -0.253196173537693, -0.9668916821542897, -0.29767477118229124, -0.5044561545989269, 0.16628170138676887, 0.4346732133149017, -0.39017260757847355, 0.06280230478856627, -0.30533266467702225, -0.36800661938029916, 0.06285324230394904, -0.08438654458854683, -0.3610115564544146, 1.0928122166772904, 0.29286213046575504, 0.23693961493583204, -0.22387151877246553, -0.1795396702531484, -0.2559000700072352, -0.21486537086748383, 0.682491276752858, 0.006452489721033402, -0.23594960298401324, 0.2158231174294307, 0.047021462892572384, -0.1440094871872363, 0.010246494254679408, -0.2593774335461852, 0.014138714577493845, -0.2702424698335753, -0.3640870195200231, -0.5694515550539483, 0.3943817739616008, 0.3154799084914474, 0.20736078839768482, -0.06224517852104336, -0.4972298009381863, 0.03709500394681804, -1.2819434021482943, -0.1429436063129822, -0.12231293361806268, 0.41258412307493214, -0.18477582017791805, -0.023311753154258718, -0.8571324026242646, 0.023608955871814774, -0.3922130108538961, -0.7637829885004075, 0.29016460936106603, -0.1840644692268392, -0.3054716871760583, 0.08215685768961378, 0.34757314271712847, -0.1152246495024163, 0.6986988751355181, -0.002258365575867641, 0.026544214748512866, -0.23815704100645219, -0.008412112802046961, 0.17077637999372056, -0.0433562978760703, 0.2051595985424083, -0.5720532794582606, 0.054066709155225226, -0.07392432916843708, -0.12016884722896551, -1.9290703081657827, 0.088024149733532, -0.07083884346777974, -0.19875064600846498, 0.1627571727864528, 0.027705427811854207, -0.04018969520213474, -0.11152433410434248, 0.13891711335809104, -0.470018693001418, 0.08963823679387013, 0.008844750504884032, -0.39259840134075724, 0.05123826714670325, 0.030379045801920242, -0.4174884401897238, 0.2801478585609914, -0.25556351755030415, 0.16299189843048642, -0.40513354299309334, -0.05233576419924645, -0.13180079199100989, 0.027694744597951942, -0.04118314386464487, 0.15437075335138856, 0.05597029613132895, -0.3364828504147892, -0.10762189713447752, -0.03866865300250322, -0.12127521976870428, 0.30117046785964485, 0.08894616076386846, 0.17838544136914178, -0.025775213228987976, -0.00802715966394335, -0.021917719087445577, 0.005222501870044834, 0.1733285625985835, 0.01239135025015741, -0.13589715041516748, -0.4329722649050764, 0.006629843870091404, 0.013930209762709247, -0.14272189577238184, 0.17775339649220948, 0.08249348561017261, 0.013002614858968678, -0.12355847106119804, -0.1319796760781471, -0.4518252488842275, 0.0007465689312025421, 0.06971501547230241, -0.5925425133511879, 0.05980470311604575, -0.09348652737758646, 0.3315194471008319, -0.011540913011043093, 0.1441642095942232, 0.05327111325276314, -0.18441743989030468, -0.523001003488971, -0.017933488376928437, -0.056205362278747166, 0.4858922658334707, -0.10974949008915844, 0.09740912966022715, -0.03932267105146929, -0.06963708572422651, -0.08635672366399266, 0.21451621393810563, 0.8001625270118877, 0.1440672793126692, -0.7863941577876487, -0.15332339712191753, -0.09908260069120749, 0.48090747001882816, 1.1037120332083263, 0.09736306207385341, 0.18509860163364983, 0.14715237008983112, -0.4357443697989199, 0.20956391082527276, 0.24788309215717907, -0.0960950221073088, -0.004805325452633399, 0.187684065571974, -0.0792720676732859, -0.0510620032410728, -0.22893769231300024, -0.3104950184277084, -0.014515946595734712, 0.5515983797381876, -0.05116951653972032, 0.17910195314744504, -0.06471463184618716, -0.16549999208030258, -0.014236293684601045, 0.19237502677609802, -0.4787159153542264, -0.10842867917310797, -0.08093227213476065, -0.09783676663143061, -0.2496427968273211, -0.44268938258048757, -0.22764776387774346, -0.11214981629475378, 0.05064499907934655, -0.24706744815032217, 0.026453218640644083, -0.452499886378602, 0.06343751992731936, -0.03388533530888504, -0.6121434257142608, -0.8025803659832482, 0.14871692485880897, -0.6554527012210498, -0.7327441038474117, -0.13846957594686862, 0.18119276067686596, 0.15292692771584682, -0.6654280545601774, -0.08253268599962721, -0.24752723918302713, 0.41649947222478406, -0.43883686839171565, -0.002882612238081896, 0.03800019708329766, -0.369432406196853, 0.049367048617962, -0.08136021478073027, -0.09792756737746182, 0.23570430606932163, 0.13776166210819746, -0.07121070454796581, -0.06858380959782097, -0.014030094690668184, 0.2963080584761396, -0.14872664892526988, 0.2633239913397135, -0.06399392110916316, -0.30634778358802306, -0.052461354983858205, -0.09788525294966684, 0.08707100165274391, 0.3615070781631229, -0.19539876778439674, -0.11166434079300808, -0.02413424056692549, -0.2809529801081242, 0.01937915622719657, -0.141643226677846, 0.024868946337441127, 0.2540452311871977, 0.19504976144336025, 0.1816855102829781, 0.054211528436119226, -0.0725568706461297, -0.08270618960943518, 0.2815290788438053, -0.2570355288664277, -0.0910940876982784, -0.16161018587373607, 0.21062988966894042, 0.19195715123795998, 0.2433019965278489, -0.01761475862549438, 0.8853834238831058, -0.082540590957947, 0.06720353388028329, 0.1570019580605831, -0.15434342196540415, 0.16248607271810345, 0.07136875465635219, 0.17195829027227413, 0.20531081080599423, -0.09827827993289076, 0.15418996175009186, 0.3188315342240348, -0.19685218097445817, -0.28877256594342515, -1.371055492700186, -0.009182039004476145, -0.5974135180232214, -0.1293107006057434, -0.3454634437111103, -0.31195888546405076, 0.2297422939000962, -0.1505422727197206, -0.252246875463299, 0.5394083469011388, -0.35067210113410613, -0.3389618055057426, 0.048288715545903776, 0.024419105636954364, -0.01959152065561884, 0.13309598613387155, -0.09250427120699324, -0.07792638002266611, 0.08072082090829556, 0.1321476918897336, 0.005027612241546091, 0.0620321674184421, 0.1033458706050113, 0.5134854708699578, 0.020156980849834118, -0.045410181079890186, -0.22670983180427073, 0.27256912158122804, -1.1239436983746496, -0.025464836599985274, 0.00672790503804872, 0.09247793775641162, -0.2824597384705835, -0.11556633453912821, -0.008260503617841978, 0.014812725013104438, -0.4324855259980901, 0.02953023919995693, 0.02191734285123322, -0.23240869442066042, 0.09214064267892313, 0.041170211730584924, -0.0343984034268271, 0.2875259343762243, -0.6486483176861055, -0.23733437197967047, -0.3703889890303762, 0.291939407490527, -0.07410470182458853, 0.05571366207302913, -0.5109881781049594, 0.07097701764068105, 0.03146311116572066, -0.01937580087789918, -0.4755565866454481, 0.04842245070487957, 0.07383487397713319, -0.12979791601620352, -0.011800279352149159, 0.19520978769792643, 0.009039817568473054, -0.315935555844874, -0.05846712667031176, 1.0149521692163253, -1.4120898110959728, -0.46239562937060735, 0.4819001808453527, 0.35059968243981876, -0.4012429722090057, 0.13270353715614241, -0.08699757379686511, -0.2836025289932873, 0.25927569426821523, 0.051273193135510306, -0.0589616409836777, -0.1879605893223504, -0.01719696895347656, -0.042207427906222054, 0.14775867881711724, 0.8279277093666221, 0.033772689347324206, 0.0038257099785378905, 0.12959234905471867, 0.2551774857648193, -0.9474653799070434, -0.03901301070079282, 0.07583843227642677, -0.2561871451540717, 0.4047970765178117, 0.10448746549358572, -1.8941420602001242, 0.3396270884957846, -0.012528896951842316, 0.0861984245994673, 0.01671322012850404, 0.14298353579965456, 0.12315596186820982, -0.10277948747844991, -0.14462615944900972, 0.12976605950593945, 0.08853750220426558, -0.44099786590452883, 0.17273497654041844, -0.020655869146000334, 0.0308060279007813, -0.2679553906961594, -0.00409415542953326, -1.5080181968400035, -0.46799188790483237, -0.07633554746603799, 0.29386283237397065, 0.17640956616011566, 0.6561986095667957, 0.20527912364326914, -0.15668513905325632, 0.2542945873221718, 0.005976077659217333, -0.029318089119544064, -0.22572263469350712, -0.20016861940355643, -0.21270348821176263, 0.1536286672303053, 0.012254956712712971, -0.7848009624202286, -0.09304606375080211, -0.3097710257734238, -1.327636642097078, -0.3298964471639314, -0.32995549318554207, 0.11762966710225024, 0.0591568650710035, 0.12456156383551317, 0.06667240143116468, 0.39454714109004346, -0.07000076815762145, -0.6563525757838427, 0.08628637535184205, -0.09829102453742111, -0.08217022926263122, 0.3174920340273362, 0.23435116357427846, 0.22020237798473324, -0.19416680963615338, 0.1215870576168924, 0.05990496124893503, -0.09417109545988152, -0.008095410599213775, 0.007191212713040693, 0.354127502799064, -0.005272075873003335, -0.10476340488781788, 0.1612839372993588, 0.14238896590761765, 0.050768887038647605, -0.0749620863036675, 0.9365770546828792, 0.133086314770766, -0.15202209134597405, -0.13107382677424406, -0.08047596608652983, -0.04435426323154942, -0.24342752378285226, -1.435045280750087, 0.6342694802171961, 0.09720228508471981, -0.19834182370688372, -0.30994630583389526, -0.5720790686766828, -0.904002380685781, -0.5086563668330935, -0.20819702309526553, -0.06398493067882714, 0.0877622793166406, -0.07931722909651803, -0.6016987482510437, 0.6877475812864634, -0.09456808192121682, -0.2762682988227247, -0.07357577679705728, -0.06247060045659509, -0.15554463193310036, 0.1502085652631901, 0.033482025439603706, -0.2864929601976295, 0.10370821882807459, 0.009068920723841401, -0.024632496693926115, -0.4424745435304972, 0.09985386618301013, 0.24448531197382184, -0.007678504874067056, -0.24929137147688551, -0.4454724357667673, 0.2682843262285499, 0.05927335690387934, -0.09583301233631171, -0.9855591105942931, -0.2944378449832017, 0.6382402228740217, -0.07984362065695502, 0.007699690590202205, -0.12777483784133253, 0.14565857658437734, -0.8666220504709737, -0.13287189557434614, -0.43352409010830634, -0.040799660243806116, -0.04599977010504256, -0.22421822440720188, -0.17986966400098828, -0.3680708072865119, 0.06204181926684086, 0.6486884723111331, 0.017947701303189314, -0.016309745801983445, -0.007841975889374665, 0.4314891629367514, -0.07269392767142036, 0.011609902106576244, 0.4323602398855082, 0.185899144592467, -0.1303889781546765, -0.4217118230525216, 0.036985216399535434, -0.37236118957189906, -0.004229746172522571, -1.2869367310368895, -0.8452819977883784, 0.0044267819713349485, 0.21352910917024084, -0.24402354996238024, 0.042145457791523594, 0.08537624530609074, 0.22179115135772282, -0.41223500056153833, -0.00011287634316328857, -0.16560516675367293, 0.2422577504039348, 0.13600545980646378, -0.1811756792302609, -0.6206469807066358, 0.03178061140482854, 0.043066257350531406, 0.3107686479260682, 0.24081912326801644, -0.7272290314911191, -0.01913283040230419, 0.08415070612065266, 0.25910216121871826, 0.061703704892364736, 0.17511621967839303, -0.16235079006861905, -0.44568461467067305, 0.12232291672011748, 0.10197024751704822, -0.2700171524642453, 0.10021037199506584, 0.1321971596997753, -0.09770639989027259, 0.12423323122245812, -0.033533463079244416, -0.1173023135776873, -0.4992428499013785, -0.3002827452257906, -0.2484233063830544, -0.34901575481289016, -0.44584802414431024, -0.2048284565398229, 0.1789610044092912, 0.0493512794705323, -0.055633729424385076, -0.2043374480962639, 0.1911973524840301, 0.02077057102107353, 0.05856875750797063, -0.5012389756178903, -0.15719762879084828, -0.15230702017372935, -0.05617338545857705, 0.04337079047556917, -0.11776609413757909, 0.01225537471344834, -0.7139943570006767, 0.19119138993255674, 0.02238983700181339, 0.05940130763207176, 0.09331603458314346, -0.30242583102911286, -0.15664991606054396, 0.004680416456934652, 0.021026420692745344, 0.01067612821662122, -0.7632428265277741, 0.09913763054268804, -1.812902023452413, -0.12629830838110329, 0.04513290017308721, -0.05552576926262827, -0.2813153872209906, -0.16933911211446645, -0.25969012469226266, -0.25194346303658677, 0.12492461458849503, -0.20411034511763815, -0.044563523271248504, 0.6866922669243704, 0.023284819473328593, -0.12106911283374683, 0.07235335815433656, 0.07370324721089962, -0.06275844262924514, 0.016114463329148575, 0.07323142160542515, 0.7597866481755514, -0.005498855828690864, -0.14390608787346346, -0.49001535168987637, 0.15043302239679449],
[-0.762892244294854, -0.4359044728501229, -0.13636524767887945, -0.3935198652380629, 0.047879752507300206, -0.10208176964210264, 0.5061656447887278, -0.023689432641490406, -0.07932707013313058, -0.10964508616252468, -0.12577340755456487, -0.08162026736240507, -0.07878691210281505, -0.3101994697820233, -0.1660765271897881, 0.09313453555422418, 0.10933516698579197, 0.003833020586373044, 0.19674904404775825, -0.1184616707596534, -0.26378285423074366, -0.017146676146176626, -0.05147031250060408, 0.16015744441712454, -0.881233488291744, -0.15231748845022255, 0.009388582161381684, 0.0009308780372748639, -0.013350672059122356, 0.051844805926591686, -0.19710204271040296, -0.7457321591469342, -0.7304818457972256, -0.6864258233690631, 0.05467764118824085, 0.059301855451665386, 0.9987489120277937, -0.6321176321480931, -0.3750399390945145, -0.5109820338835542, 0.7808350528347884, -0.105453255386212, -0.09772795643355156, -0.18990237372960572, -0.2361786958443668, -0.21587806700813666, 0.020907934462390414, -0.04829121220502626, 0.3925365894162843, 0.8843148290568318, 0.1635646874338668, -0.15871568166498695, -0.23429079763151048, 0.07673290472199366, -0.031539997872117316, -0.32721405162714756, -0.5648295280230786, -0.3285202871174683, -0.19581268071536553, -0.4274137775261104, -0.09876752269247684, -0.04252988297347333, -1.9177218036680472, -0.1483919408649738, -0.24272662986519208, 1.419403000312371, -0.48423530920263963, -0.0056265578697747235, 0.13389276673388648, -0.27876004790745323, -0.024589894120907774, -0.07790267283782196, -0.3118795578102576, -0.13197638801351186, -0.8212891687608906, -0.21882706370666813, 0.028859759778111112, -0.04690120222872025, -0.3900758855729253, 0.03634157983064368, -0.33633486076279107, -0.08929001570717374, -0.0909280369754691, 0.5260150951731618, -0.20076997818530345, -0.09703241667809571, 1.139936319388177, -0.033275956149508595, -0.3608423676533554, -0.29244346091574447, 0.3146665649584409, -0.030622629243829403, -0.2521840265786442, -0.8307097094420507, 0.023193200718701252, -0.06951297684184021, 0.118607667360987, -0.054121335775913645, -0.16403830970797129, 0.01399229091534142, -0.18276729934044728, -0.17013269820731486, -0.7403451547244446, -0.14473162285433228, -0.2353432432127094, -0.3142114013463902, -0.1142543214296369, -0.043380944341345366, -0.17172544402118461, -0.058292340843823266, 0.14520784227865768, -0.10645258071055129, -0.011513700011706164, -0.3608291755909224, -0.009612851666061883, 0.09693057370999836, -0.17577199115245362, -0.6618358125328244, 0.14790130221842357, -0.3284769504943438, 0.018379258080702728, -0.24085720522016224, -0.16752590656009783, -0.2509364087708709, -0.28371887792560085, -0.30256052717952153, 0.08071059206210363, 0.14880256262893005, -0.1292653379939582, 0.3274864946756993, -0.1253290243919567, -0.001071227795583736, -0.11498387815984817, -0.01279640438307762, 0.14120245119856878, -0.11251667932461323, -0.0024387207540694403, -0.1128355664644315, 0.09987392164394272, 0.02408390477822417, -0.07730735644922851, -0.14695852804252554, -0.2107956429923367, 0.14966812907888938, 0.1653310801600113, -0.32807827373853343, 0.7358597313600485, 0.12666332964282798, -0.04665415514446229, -0.13488375516906337, 0.028358698767900894, -0.35152867526991255, -0.15896105049505252, -0.13955513703173975, -0.4100174074466098, 0.22563931938886203, -0.07272047029940665, -0.18823014595852477, -0.22340182785530432, 0.07980787650167533, 0.06818818334634988, -0.36408638268973853, -1.0701967086273967, -0.36939456235609425, -0.20573579140663328, 0.34993682972751905, -0.11181686731238531, -0.0021955675745197263, -0.11421393320736914, 0.04348928146008927, -0.08555224778566466, -0.056287358823349946, -1.3148123678482828, -0.03847963917269233, -0.7595217733222356, -0.163642515349902, 0.018081625101298578, -0.11620116607333655, -0.8519819404736004, -0.06506025771815549, -0.04033519511499189, -0.0297321216373236, -0.9435119270508168, -0.08587116297617452, 0.0570252096817489, -0.08708672665960907, -0.21696498045871657, 0.018303960646220092, 0.038926880194280804, -0.11284763347933816, -0.47986977334472825, -0.229907036345656, -0.10961259103523312, -0.23001807542588268, -0.028452832106426445, -0.16951711553701687, 0.013718735521779964, 0.08499718087415456, -0.11133188456379244, -0.04308004337312454, -0.34329940442203494, -0.3355177287250735, -0.13616133425486115, -0.11162511015454947, -0.20159573509741172, -0.2435976163650986, -0.15568674993983542, 0.16048580385831906, -0.28654214782594234, -0.18010962867700533, -0.12983242301861125, -0.4235225912462689, -0.5488440008153745, -0.1106725145850897, 0.02173714957192161, -0.48190171083680144, -0.0700524169428008, -0.06813781824952966, 2.247744356599836, -0.19077697823803275, 0.39670630296594245, -0.0728457174223028, -0.15702499218014096, -0.2876536821392495, -0.07585984952789683, 0.9933438112983323, -0.08258253341433584, -0.10680847652637719, -0.007800500496903772, -0.3700644714162623, -0.05853196266219234, -0.056429370800598865, -0.17376059251642162, -5.818842279641344e-05, -0.0016042656838437222, -1.0737048543501206, -0.35559583811335466, -0.16218049082853905, 0.4147780355570797, -0.04990373675050211, -0.11706570213143673, -0.6472783982664914, -0.1664496016195238, 0.04482473746860533, -0.5299961202840943, -0.14345148687028964, -0.036580942801911265, -0.310098913209857, -0.5119927437504908, -0.2855235338226821, -1.290328636498895, -0.2564779056067049, -0.16301307086691727, 0.155096100698745, 0.06777933421840118, 0.002042987321833269, 0.05487831650658857, -0.012669364521166234, -0.2009908361675896, -0.221844842415472, -1.0157539198317365, -0.4739499520781252, -0.15918880733525512, -0.07967636627648637, -0.10895623666177903, 0.12737045942207448, 0.5126428039925741, -0.0726912871721772, -0.0411764830395658, -0.2502618905333859, -0.07651691965162108, 0.0406397189009075, -0.28162809732423294, 0.002602381459930553, 0.10258821671120076, 0.13948389797387575, -0.5315500411203192, -0.1998910970190983, -0.10022445565523752, -0.06657705514316518, 0.21740420952384842, -0.18091062732742508, -0.8160908314623625, 0.06912084219206557, -0.4355368619448873, -0.02458397804349127, -0.13724684500448853, 0.12936130236637836, -0.14071109494512543, -0.2048181051465463, -0.07952708168332857, -0.43129898102795344, -0.3557812762910175, -0.1209285820853958, -0.021234430670881523, 0.06340493666260219, -0.16995966215284675, -0.11179238264284928, -0.21373592162468555, -0.08014937435769712, -0.4255190094917578, -0.2453051604180788, 0.0014116961227194126, 0.0515159015420625, 0.024735914586919445, -0.8307885547699302, -0.07186994910722107, -0.20866438619695843, -0.15720763207675792, -0.005265698701397553, -0.8114710475823514, -0.08654823114316701, -0.031171961876499617, -0.4043091035040212, -0.4906480274142235, -0.15467053427851468, -0.0697951415058936, 0.04745309754509691, -0.33345118835950627, -0.5718446557297344, -0.3521219545608262, -0.2692058921987762, -0.050941416874156556, -0.054300774685059686, -0.06033004166791627, -0.08236435284585922, -0.30115918250774487, -0.08566954405279004, -0.570479022619933, -0.4795650214152386, 0.04343696798959027, -0.25166761138305177, -0.7257653350406967, 0.08700245032529726, -0.07165265911351894, -0.0645537015745488, -0.3239730169713911, 0.05429048098453484, -0.10989314433294892, -0.2117963596676224, 0.4641702839554991, -0.14335969172122132, -0.5319297197512268, -0.11394979082876629, -0.1351533116346735, -0.7913238042244042, -0.17679700279482188, -0.04449661499214176, -0.10331723293934215, -0.3409089482190741, -0.08486548220851055, -0.06860628964301652, -0.20270855898628273, -0.13390323141798272, 0.022779011947372738, 0.02163649657219888, -0.05883029122086695, -0.19266138832455718, -0.04714483095635262, 0.17209864648138787, -0.023380049187771125, -0.3960742531250984, 0.05983474926728843, 0.17027470055074664, 0.02510142166852286, -0.1291031263475403, -0.41864464420959546, 0.6844644530892634, 0.2882289194251409, -0.05787454173786236, 0.6394333442160712, -0.004445786992371463, -1.3319154078621536, 0.1748048566658364, 0.10967398507034606, 0.15664805247072655, 0.01010080252480525, -1.1577858144172748, 0.044026540276707236, -0.20049809364846188, -0.01626857082671135, -0.1729051673834237, 0.10024878458706787, 0.4419539935414751, 0.06975441559231285, -0.2352002517005162, -0.22520051088944065, -0.04674119237131164, 0.046269534603763054, 0.9122757523365227, -1.103587133793321, 0.024424246920863094, 0.06322743084494264, -0.11005917986238853, -1.0513750409022533, -0.12762378040024527, 0.12056523606366301, 0.11449363078637574, 0.061375498607971576, -0.12547791466472427, -0.5511476136033018, -0.10830469602412883, -0.1299866069796117, -0.174285458942903, -0.2465859546486262, -0.28928049484826646, -0.04798850807269864, -0.24028691938410907, -0.7421237048835401, -0.5190730430993982, -0.18956300537480522, 0.05306702426570461, -0.06555533857672896, 0.0603399863150333, -0.06402134198722174, 0.04128797475925934, -0.2700280785410247, -0.604091014714786, -0.14731321340330575, -0.0648820140346666, 0.03678535663515583, -0.03370782018841497, -0.024954737008542265, 0.14813799484607704, -0.30349239794873495, -0.19647025697327355, -0.24332731192069976, -0.09532105195589323, 0.03538607198162944, -0.03457533492291325, 0.0797976392801768, -0.06735178407067088, -0.10326430046464613, -0.24321842386459613, -0.15119326442224523, 0.20469845424031904, -0.45984464463029473, -0.07745337463076951, 0.015554225097849731, -0.31935615067268425, -0.06779793207583062, -0.07507916338301823, 0.15049198762033708, -0.1568653232343193, -0.011665760197846336, -1.9136768438407912, -0.19329699711382875, -0.08696339356212028, -0.11564141972222249, 0.057001346018810045, 0.06596499897458466, -0.053930401607027195, -0.2946903261914875, -0.09918289098667915, -0.12470777742754077, -0.1448489251833194, -0.9962235707961717, -0.6480092591299274, -0.060771428123643785, -0.32625413492212263, -0.3990318208013952, -0.15552783076129298, -0.02545423048956408, -0.06468357376983183, -0.0900044941891318, -0.28760503516208585, 0.03420052862304049, -0.14896821805557492, -0.09568036352549876, -0.7235463994088985, 0.017730990975263455, 0.16842075060326284, 0.01323235311959438, -0.427475906847235, -0.36949492501812503, 0.009739231150753548, -0.11982388457336844, -0.13658504653025358, 0.2640141163944617, -0.1055153709757188, -0.6183504009698698, -0.1374444369196469, -0.17495325991379998, -0.22942452516288933, -0.05971563048306535, -0.9291106456345841, -0.08111358744403424, 0.07741499778094786, -0.14735071274627762, 0.10828700186073197, -0.42357798999060414, -0.012118298156456313, -0.04655844270134045, 0.08222082029245296, 0.0904283759478167, -0.011779068835924437, 0.23839619164602288, -0.2935073973003332, -0.028567922053450948, -0.1612507505518024, -0.051173413694915446, -0.07226990715573842, -0.2663791705085491, 0.03890381305352302, -0.4590078169740921, 0.13515095582831857, 0.20566401934034909, -0.02818138962647016, 0.7620445679040919, -0.18648952263282823, -0.06881180775278116, 0.06151618715710699, 0.010342269326736401, -0.5751145051274897, 0.1201559974118769, -0.10307882377238206, 0.9650334880462201, 0.047520101382769624, -0.22008910740769655, -0.2712935705320429, -0.15713913081357003, -0.5904033938514467, -0.7571722206953464, -0.08422586182338687, 0.01546123939717975, -0.4123968482973377, 0.028440309288442777, -0.3298054972473365, 0.0605278878130558, -0.09531037981024804, -0.04182034184978889, -0.030198848244616525, 0.022871288842464917, -0.36772543552913334, -0.044730466979857435, 0.09679014298507614, -0.14364506695723459, 0.02123976547047469, -0.018030404157495638, 0.036323055522090586, -0.5232052338572931, -0.11278292469405975, -0.09045276673836357, -0.18264738687267246, -0.05040890435541429, -0.2653873670228768, -0.44626674572293096, -0.27084458435637254, -0.4734495776219782, -0.14470914758776968, -0.057496299823813136, -0.04080663267027364, 0.228032423386268, -0.06522325050922456, 0.023117943744484896, -0.11060914630396679, 0.05125665948887269, -0.4705622790752564, -0.0411120898064994, -0.16214810911277946, -0.012774094806507116, 0.030660319384802567, -0.17229048953493245, -0.04432183726626225, -0.03770035145718638, -0.05941938995441224, -0.8122196706021555, 0.05201782902398329, -0.12686433085790075, -0.26205753678816124, -0.035817243847404215, -0.09792065284443688, -0.2316317214793318, 0.004125423286395325, -0.23444892971196002, -0.03275033926635271, -0.9480948870432401, -0.21629362161476523, 0.06980306676694076, -0.4577407503782422, -0.016300238405459966, -0.026935486146302208, -0.30122182417336346, -0.1213436532905276, -0.07992999745650958, -0.016630687209882287, -0.4657400483016062, -0.05487653093695661, -0.010997964166988068, -0.09356530504409341, -0.044042362212205574, -0.22735166484585412, -0.04162066438969493, -0.09352807876731027, -0.13301035550718995, 0.09711386023838214, 0.13207096606184762, 0.12149927655442569, -0.759554439289995, -0.397877886157941],
[0.14100232978297783, -0.01541237533087596, 0.11070104787146039, -0.4465291354954417, 0.10557656020456546, -0.08503080599193524, 0.4415990770001594, 0.1465268114429399, 0.12216111141697972, -0.2327784556303243, 0.09143986028945028, -0.1327549782231971, 0.2995981075115833, 0.0622421304564747, 0.3282478294230013, 0.18331137658544838, -0.9916767683917835, 0.05732364408019079, 0.1880979843205964, 0.003839656874734572, -0.20904648506114049, -0.2008749781821089, -0.06350655587394786, 0.36471958397751264, -0.6603734992626242, -0.6696646374658353, 0.504365954178925, 0.3994788594709243, 0.01254013773406199, 0.14425656481376406, 0.13827534724232268, 0.023726859355328426, -0.3036259887196344, 0.011679579819892198, -0.1323825829681493, -0.05680639521846163, -0.16155164054378507, -0.21514717784220388, -0.2932762256856868, -0.25997154994603217, -0.32280112295327074, -0.423976948809485, -0.2947946968015433, 0.02974235556286447, -0.06208923101479379, -0.2793814462566759, -0.26690792421956794, -0.1994375480685585, 0.2513466280307661, -0.02288437259161938, 0.34527356265671444, 0.011547920943123683, -0.3212548229423512, -0.09282405919272843, -0.08676381825010353, -0.8357144572540713, -0.6515268457770377, 0.15875767182602388, -0.09792399527867326, -1.5519975355447257, 0.16635422485762077, -0.3419964113237616, 0.115570523689806, -0.41367832599169524, -0.09655389213347829, -0.5992894739150166, 0.36029526034723525, -0.41775272037916444, -0.005212039370063866, 0.03919868000252772, 0.2960832752337764, 0.08876376934590749, -0.2000438126248908, 0.2881965714255618, -0.9876364891091728, 0.0430582800377127, -0.08958136562123026, 0.0714223409642708, 0.13969391953882296, -0.24448911013832486, -0.08534816249150082, 0.027617400507028714, -0.20105118854577522, -0.44803137628359346, 0.169229067561372, -0.14343047967838732, -0.21809736307789596, 0.2718685825921439, -0.1827916927997978, -0.285756704345922, -0.6185397325213327, 0.2712372600340023, 0.18399069910505592, 0.37633902716336004, -0.07657120819433592, 0.15589589754798977, -0.08401302835224062, -0.07310078499557553, -0.33116504625167265, 0.1846405766262045, 0.011695216588097983, 0.07084815848499615, -1.1958554634200629, 0.04089522393583156, 0.02370975859766556, -0.11180000485369511, -0.15386707069132002, 0.32077087732994264, 0.09806782474389497, -0.2990921655413106, 0.12114560130574054, -0.22349747057141223, 0.1446596741720122, -0.042006898747055495, -0.15672136447295185, 0.10222726971915418, 0.05901144362041472, 0.526806068273518, 0.042573270528213955, -0.039029856601925775, 0.2276112201301151, -0.14286995817522288, -0.04830868939355223, -0.16948985717730755, -0.3345535261335228, 0.12632736844479928, -0.0867069898200896, 0.5303927271786955, -0.12067908338693929, 0.13528428053830016, 0.10439641883735533, -0.24678016171315592, -0.5745775277027961, 0.307231142845223, -0.020568472018367426, 0.1918917517966536, 0.006806952008157234, -0.1486489017146256, -0.02940414070379001, 0.21245597422961557, 0.2679295651095675, 0.006174438010320864, -0.5303145425537006, -0.003239361532186775, -0.06916812052814517, -0.22406717582112637, 0.2831343577860298, -0.4856460673139993, 0.2997078269214241, -0.07054733056321265, 0.08631939334166422, -0.2968713329752704, 0.18517520740811647, -0.0633580921074615, -0.359832145821809, 0.1455461839135357, -0.19856853898382482, -0.092935465193165, -0.17663403175983874, 0.08325204616226632, 0.027981627040054313, -0.08689096380813997, -0.06752170674600219, 0.1908509837572272, 0.09770770165315985, -0.27188535244557316, 0.04772135712574334, -0.028716999321171516, 0.14742542247961088, -0.21104817186680494, -0.00792020001391173, 0.10215876977185417, -0.44430563501495485, 0.3909855679801297, -0.30419704928249075, -0.16332663335364414, 0.8030832110258987, 0.5100400746045971, -0.3904993617158531, 0.37863123208314864, -0.06611813586892876, 0.08302405424497696, 0.9900813789358616, 0.23562782208215532, -0.04281395975261747, 0.1660419049610286, 0.09405136927772795, 0.3129966076500773, 0.4188861048764931, 0.007058149210840497, -0.22796855844198133, -0.06795478030554068, 0.09954963206439134, -0.41061352682392005, -0.12039654502029362, -0.16950565344449844, -0.22886272287906462, -0.1294239611356619, 0.1138046401538912, 0.030390747682822714, -0.24601417702597184, -0.3311269892608762, -0.05031329742718912, -0.2842702233785257, -0.11400768077639624, -0.2067365007064427, -0.30132006547626755, -0.09023593589952306, 0.011307721185075703, -0.24040933300621112, 0.10786942077231013, -0.3513506063863766, 0.042243284081225616, -0.07785777768158217, 0.2561837603609132, -0.545744736949953, -0.09966968082984229, -0.03948489375726883, -0.5772837188865636, -0.18117186312527125, 1.1332193685670802, 0.10637024651061747, 0.01868804728506714, -0.14724109643943858, 0.1434276324629424, -0.36089991224264073, -0.15745532229109735, 0.16887976885260933, 0.044135689665456126, -0.09850910418919986, -0.09607018143104004, 0.05408973983250777, 0.2729160402771384, 0.10356258609752676, 0.12937082123529886, 0.15581644017793977, -0.5017106363464378, 0.2723089574880958, -0.814204389763319, -0.1627457598010017, -0.4238134516921863, -0.5444545954387084, -0.14442740066264662, -0.014149824210539532, -0.40378359597527475, -0.26264160367135103, -0.008373199311269756, -0.07179536588298192, -0.19406464788098687, 0.004166889669655142, -0.24308194285148457, -0.259296225447701, 0.030619225520729706, 0.46849077627849756, -0.25910019569954906, -0.06525800509993245, 0.06600846451998765, 0.04188440396783683, 0.5122425411960199, -0.22131281282206616, 0.6915411344110388, -0.5602133122523529, 0.15116563613417275, -0.09932499739797693, 0.08742976874880394, 0.05051318816530228, -3.5790332581860294, -0.05915138290284439, -0.8934757165515552, 0.25743027012350905, -0.11252388462347088, 0.10715897455123853, -0.21162228796806695, -0.1631249364914, -0.2204197985102154, 0.30153080561680823, -0.5107261799879911, 0.1468613047390484, 0.13704462384326904, 0.07906300717945383, 0.26909553475125497, -0.08971921582145416, -0.7389725345582594, -0.045613478958857055, 0.1520128641465598, 0.060604606883397666, -0.043835838361554406, -0.4187183486797775, 0.23730312961388836, -0.14560980036190865, 0.20452589202693047, -0.5925035476785669, -0.04068995824962345, 0.43170644487863324, -0.051479224938531895, 0.2979992383193905, 0.04245911556773734, -0.17262635607790505, -0.11478278477472659, -0.031329041077497816, 0.2672730791422989, 0.04511947295296283, 0.15128400944008905, 0.31255674724183136, 0.07387226442985388, 0.32490969574526685, 0.2064699784206835, -0.021848196431278298, -0.2619693220792512, 0.2249137185959305, 0.26001815467971584, -0.2300109577582105, 0.11855111587289532, 1.2411375908833016, -0.40482872788138347, 0.06778809945656324, 0.015478878284821225, 0.018152430172072784, -0.07697511748316956, 0.11428474644835755, -0.06203201557939017, 0.06924957716016879, 0.35281634116182214, 0.04132622224984415, 0.15814971152471424, 0.2680965009575137, -0.38610878816318184, -0.04890570548937179, -0.20002006139625852, -0.1336030755645793, -0.13322189962530945, 0.11109698470638438, -0.4617591967729481, -0.10855304286843173, 0.21353722668181135, 0.6206702509706714, 0.32964232742855915, -0.26262436134481265, 0.4518059932146121, -0.07270428502667335, 0.5167149263462787, -0.07153418189476331, -0.04958910478185894, -0.34151810085608025, -0.2908099157048257, 0.8821095266943344, -0.6405577747297498, 0.18673721287896747, 0.5635256726314537, -0.22787381724875963, -0.22425660605367165, 0.029456589856124937, -0.3672913718118207, -0.11169601619311764, 0.3034101445512122, 0.027171057148079932, 0.2682750649186058, 0.03493774277704275, -0.11338523399666238, -0.04214263176571549, 0.2703593983267479, -0.47534851989494037, 0.014858783750147337, -0.08725403297718361, 0.18774204076929538, 0.042042012740397754, 0.8363441746998609, -0.4664035609069301, 0.05787276487260338, 0.04201965809120112, 0.3427104201110941, -0.21812911029403043, -1.1217077929295638, -0.3884422477635127, 0.0761771797840209, 0.37278872387968, 0.5239534752879957, 0.3760603894194678, 0.6548572761088194, -0.1729310760586511, -0.17440028827209172, -0.35881352329067356, 0.2668374006737169, -0.5736787324179484, 0.038231199252560924, -0.04863446818771273, -0.21645474808278248, 0.06942752222589907, 0.09456656554247292, 0.5344499890405658, -0.271193792075583, -0.18714529372373342, -0.1511382507345306, -0.15439911527029415, 0.13017290091728836, -0.03621863091343153, -0.1496883701914799, 0.11519949436923724, -0.04603715437828056, -0.19032987729049475, -0.20251753798297964, -0.11880037627974528, -0.08812582339914143, 0.1026086333487326, 0.005439194078727013, -0.9625800135395522, -0.36945487063277677, -0.23475448259123582, -0.4711500186804493, -0.03913439123612569, -0.3388959053881819, -0.1336411635025498, 0.08968935303250211, 0.05755904216001383, 0.2203914684668071, 0.15379727542401067, -0.018167090220563946, -0.25151551641857134, -0.017394289305825733, -0.06331994348924098, 0.18944811637121262, 0.13521343780597453, 0.607001266423376, -0.20287759565419522, -0.19090488708268166, 0.014872142039633928, 0.01566078119927954, -0.13075261479664224, -0.0657672152752133, -0.0766713899064255, -0.01787725976373144, 0.34134965900263964, -0.08697520117562298, 0.28385478786548807, -0.07165034394987738, -0.46389880363530017, 0.14395070742929292, -0.18922221087518853, 0.6139789736412176, -0.018792571771519443, -0.0590144194728873, 0.011253841505559647, -0.4996845701496562, -0.16522157673367255, -0.15433150062066864, -0.5292995783174278, -0.2057990780938556, -0.02068999470610602, -0.021022258127500183, 0.3845110144894287, 0.7693437065106526, -0.16851060549738722, -0.1945963346709421, -0.048872343687026025, -0.0026189532623842075, -0.14963598631749497, 0.2693485400023938, 0.007204476004781369, 0.13883993879138182, -0.018120386411069632, 0.06371917527249782, 0.048759534225771985, -0.03828415959524398, 0.46644186810993593, 0.18215754475061546, -0.3110216319615171, -0.2117330011985991, 0.21861509347878, 0.1406493159949091, 0.912829800518693, 0.22051822189574688, 0.04589853960351793, 0.35695319635789835, 0.054204420945611856, -0.040718790186059874, -0.13711935584982748, 0.15959171134551037, -0.1877393692651581, -0.22614678931957477, -0.343286867872232, -0.5229522527893257, 0.1269377967527434, 0.041969739740410975, -0.1875665967393186, 0.04374117861828749, 2.73344050384597, -0.015670486493764343, -0.0041238966858325385, 0.00354285524198608, 0.08861003196931334, -0.04779679599677372, 0.3497715030094618, -0.07883709301962538, 0.20431849825020687, -0.4640644301375298, 0.36263710506033586, 0.2822130091730323, 0.14926604204584148, -0.20585863101814617, -0.32906679205687184, 0.21773317668019473, -0.07564030343898509, -0.18810230646128967, -0.005378562494211664, -0.5851348404190971, 0.0625458963999495, -0.3818155975524955, -0.022585584833452174, 1.1100533611922154, 0.21379224653505102, -0.1379441472858761, 0.2545625059069225, -0.0849520136567987, -0.688071278393333, 0.4224919458450993, -0.1965920190539769, -0.933028704904317, 0.4003044206688403, 0.257626845755444, 0.5847673634747241, -0.3912313608951763, -0.5215521210957043, 1.1868080893548887, 0.4460003415382951, 0.26767654163249227, 0.5439515844826576, -0.004360891211010932, 0.3298457362409364, -0.23001968529326647, 0.4994422947354816, 0.16588349567068297, 0.40330359383657877, 0.1620017512839098, -0.08211929263227864, 0.38793141210044746, 0.6616244113536509, 0.13389793789211438, 0.10625748120022363, 0.14910229986126916, 0.20617523169324473, 0.32658124610074074, 0.2330884460957349, -0.15919652502192935, 0.12831165178755127, -0.1961484933088899, -0.021004528766925205, 0.07665063517466039, -0.07648176223810754, -0.09995725139649911, 0.0901848162754453, 0.1768145421069327, 0.08943632856891073, -0.07522403300340898, -0.02470992846502643, 0.06982642436412205, 0.05085427289044695, 0.47831517142149366, 0.3554597698717417, -0.16864298697571425, -0.3823801774353791, 0.03784929273812635, 0.35253677058251226, 0.03543793347096713, 0.053622977104473425, -0.4221346512617914, 0.12240043350959016, 0.3397219042872813, -0.21054154811000045, 0.2104510932096345, -0.21175697043807165, -0.21286940406643048, -0.09397994148202912, 0.013527892179917667, 0.37988170855408604, -0.063432355501577, 0.42779180561512803, -0.7013409501605319, 0.05402142785367131, -0.002439258054400798, 0.013918224709040352, 0.14339952310996187, -0.11208691006197605, -0.14217923456383946, 0.06670622965171215, 0.31720750559225336, -0.33090255249803086, -0.1258433449967375, -0.14631915815696406, 0.3826635215355113, -0.11413540310587865, -0.39501242154430344, -0.46839477928396595, 0.0780396197949793, 0.39458639064931117, -0.0075211860509371556, 0.36464265003045676, -0.36371211110771734, -0.21070988544857058, -0.3663612787040683, 0.1349011975344455],
[-0.973305761184611, 0.053663620327035304, -0.0858894104558316, -0.39626041591711936, -0.032705808676139744, -0.14183092968349054, -1.7513628299375883, -0.0811049355075616, 0.08263016124109597, -0.06916395185194439, -0.02319955802366694, 0.17013776287446603, -0.32271590628592917, -0.06125012360374863, -0.40402825545787385, 0.11908488442045999, 0.34088442373766314, -0.26785972167037003, -0.12206799600680511, 0.038757155516229796, -0.13356620874558692, -0.2962426894454984, 0.07709480212312905, -0.05358158488230386, -0.6936120421004474, -0.15820711913394525, -0.11163840577363363, 0.02888041686422937, -0.20730805271409997, 0.19359880484346892, -0.4818826238104541, -0.4155625140334541, -0.03668404941666221, -0.2638078731090559, 0.08995459517973062, -0.1390530063816727, -0.34214518306236863, -0.4948282518809856, -0.3854051994063605, -0.007011754708054017, -0.33187808544846215, -0.09282016525692974, -0.502907308736227, -0.033583761493257015, -0.2922312727852181, -0.17701140172789656, -0.04781871552754998, -0.14030027698518652, -0.7484790357915556, 0.09579220600624587, -0.11681780546306021, 0.14117508240514887, -0.31032717284220257, 0.034304054848238744, -0.04198631467869617, -0.9879447356990332, -0.3244897201439967, -0.08670755323427198, -0.23013169439912437, -0.15496314317435042, -0.02381689806031652, -0.3441165469259948, -0.39674263465744414, -0.20319504061475221, -0.08207543667820658, 0.02520359817513443, 1.0422457027531653, 0.4202688128142158, -0.07089036279097775, -0.19494348519424362, 0.037527754349149144, 0.07052967048460863, -0.5582198125703022, 0.015082605024739846, -1.0614574874013418, -0.04614830242632528, -0.030336005932363016, -0.12783771213487238, 0.17083182925512425, 0.16147727899075587, -0.46571826525047, -0.048524159059240395, -0.594791353156523, 0.4799732333372087, -0.15618665112960028, -0.03775998071771258, 0.4220158362749527, 0.029113937591446974, -0.5584323563164031, -0.06908581793994233, -0.353107407822763, -0.028959207468092774, -0.0825912800173471, 0.1647176651980536, -0.18853771460138594, 0.11446958548542316, -0.11816897328281896, -0.020235046039986597, 0.24281185046933587, 0.23287731129301137, -0.1783372030677621, 0.10299383148620585, -1.0005781016486806, -0.2334516258845009, -0.2264324228234101, -0.2444090490490609, 0.04996625896158548, 0.08410326910289936, 0.028313300768219942, 0.004422495791899077, -0.0531082959294723, 0.10129516532865582, -0.06956165131633513, 1.1364544832870138, 0.2267419917408011, -0.29871827796813266, -0.09568098495982628, 0.868998030723014, 0.2106468423456515, -0.19956869502059063, -0.1280414761010785, -0.17478132907510088, -0.1440896826394732, -0.23822897606229132, -0.08091525271505151, -0.0352577159305736, -0.17739734471502439, -0.031679458608181256, -0.4099419680215965, -0.25441536745264476, -0.030377962587270088, 0.0038397721528816713, -0.27722058364115637, -0.004445786548231781, -0.6076275685516038, -0.07097319812061165, -0.18919839348022183, -0.39064832992258036, -0.05444383547114538, 0.07807857986999275, -0.04814355688832221, -0.17222735178221965, -0.03043431626508513, 0.11008584764690418, 0.07058963181166858, -0.17740882946813488, -0.27975008717460875, -1.0074395305577575, 0.03035810712274724, -0.10959607014228534, 0.2768702306959638, -0.4944465643885684, -0.07257209639239313, -0.15791897980200303, -0.7588996425347697, -0.31107248218069233, -0.09107506000220691, -0.03872251802812813, -0.5311696354623787, -0.039788802511014035, -0.09629084473605985, 0.12303957311761496, -0.17134065256272393, -0.222070146936954, -0.08013355512285046, 0.11457642082215626, 0.08833552453931422, 0.024481418937175384, 0.12847193265595327, -0.3084207150537943, -0.2280714627661471, -0.23304674930072214, 0.5606068043995538, 0.005182434766777416, -0.022169257754225673, -0.350447730303974, 0.041021794793036866, -0.8653853110523052, -0.32838259838645706, 0.011778065037948312, 0.0017974616350042885, 0.11683483261209571, 0.44977395263681247, 0.06109264199766035, 0.09601876590556012, -0.0034014341077044418, -0.05387036246766599, 0.06648392913784673, -0.1392352854495454, -0.07889842413845097, -0.4612900903985633, -0.16306442262684798, -0.2649637515563118, 0.23889167327667502, 0.10697040406870778, -0.21791217634185453, -0.5599744181284434, 0.041728215079461055, 0.06589367421339924, 0.0040142639343007476, -0.5675574884701343, -0.15287245696912227, -0.03145129450368203, -0.15230818103914, -0.1959578271791695, -0.3167117758547097, -0.1889427525550964, -0.1744504946684656, 0.1452661807802433, -0.0880691808158983, 0.026589626858719774, -0.7426733475572644, 0.03674506663776447, -0.3332223521892306, -0.1623369717271667, -0.05063334226885421, -0.05527615013924653, -0.15805909281831343, -0.665358081633528, -0.15760109851362206, -0.38875478767794985, 0.2204914080481586, 0.20460256786000064, -0.17846333011306323, -0.20401997352983445, 0.07941852895782164, -0.18196591233396409, 0.19835496563915694, -0.17996560841818943, 0.3172022086187108, 0.08225186917179286, -0.1052857651703165, 0.19625802590891145, -0.02826621471764467, 0.24542143089405796, 0.25831108741703107, -0.24269861016783972, 0.011229076718542398, 0.3319480623783386, -0.3550730421400307, 0.1138116508182585, -0.2813321462694234, -0.32609447554219767, -0.27786383758964855, 0.11511646387268079, 0.03887965723071839, -0.16199190462696386, 0.05590932219573413, -0.3917018846280829, -0.245761537210606, -0.03072257126945109, -0.5113068456618809, -0.35858994465959115, -0.07768141818385674, -0.10733545313805226, 0.06824962958936893, 0.013421840876682597, -0.1258763145360073, 0.8238870997357607, -0.06760994587399276, -0.860018689124695, -0.300854498348988, 0.009912351510041781, -0.21027509254971638, 0.01988524061463758, 0.13599152582383697, 0.37169007880310867, 0.058441100390596176, -0.14631678273059226, -0.2516419949731667, -0.13221307763331866, 0.09009250079982148, -0.07998773875269169, 0.048598601140765, -0.1180543975648853, 0.013984378199450389, 1.0603228384344328, -0.06586001570947536, 0.012191676286804282, 0.1904453252131341, 0.05216673284894466, -0.1960504896706455, -1.1119447827902431, -0.03379030555180115, 0.20863054077443188, -0.06062923029667622, 0.4436058493934554, 0.4212311645515859, 0.07028104752561945, -0.08929267139348575, -0.04851863761243187, -0.34504943094652235, -0.44174555634061946, -0.2223240626367368, -0.041801827949936615, -0.0545830098394402, 0.15110558857241885, -0.028687760167740436, -0.40329220261817167, -0.09630784598070316, 0.010151985016005602, -0.10006232608915457, -0.02658401729833609, 0.06393291954765912, 0.013164934542085933, -0.26607689489675485, -0.012458815277935588, -0.05626632529275807, 0.12641929511820962, 0.044103074883830494, 0.12332379873701921, -0.28758323301743016, -0.08868486879440408, -0.016485086935451462, -0.5347081116009377, -0.08848319647984326, 0.050917087824095805, 0.004643799667824474, -0.2065731875692315, -0.23234685915817932, 0.7054359783887594, -0.12954177660254182, -0.056997288741216824, -0.038496124004189565, -0.04486927381471838, -0.08235048591579534, -0.3407266542717498, -0.14992064872441638, -0.13461002600912741, -0.20704263062369474, -0.05405641833363102, -0.08356635194161044, -0.5888367612429772, 0.13123064068070175, 0.03876364352346715, -0.020003123021860553, -0.2593187049360547, -0.271033784321538, -0.02548051783631784, -0.21549062993368648, 0.154331119404087, -0.26942026244577466, -0.2830569435503965, -0.025126060028861412, -0.33168271963640555, -0.890053561757748, 0.5222712351012954, -0.16735676452841386, 0.6060097327305548, -0.13413200540640877, -0.08037337849747925, 0.24358381342235155, -0.2256752886322122, -0.1615345172697608, 0.02981805637727853, -0.10831890820157664, -0.14607703263538696, -0.29923739348795564, 0.09023660784315315, 0.385539893782859, -0.06583512079717085, -0.18897050151037592, 0.06721256950293872, 0.13805607518974763, 0.09868942570011205, 0.12465429805353195, 1.131372119606087, 0.14537359160502605, -0.47878776832512165, -0.19113876484517117, -2.6083418864704613, 0.020867182809740106, -1.2692643548148645, -0.023234647054764742, 0.3423432306068091, -0.06426768969762127, 0.04089736315249311, 0.00410396575243409, -0.05698947418017787, 0.3528608753450588, -0.32129445089338765, -0.20507392312936076, 0.12182842038298915, 0.4865347341501799, 0.018491946600761075, -0.2117036495043801, -0.1497376050196812, 0.0030704603253029596, -0.14615429176846534, -0.09841664470933338, -0.41634788402414674, -0.16806111381500974, -0.14304504506382038, 0.2002215093733588, 0.14077503376170816, -0.1027350022096, -0.001990181048308455, -0.0805751279037864, -0.0673350886034738, -0.13797135238505984, -0.6075512897364371, -0.17131476434357998, -0.24236724854032157, -0.041097876106682085, -0.24057026322388952, 0.6919273450465224, -0.23548191957947123, -0.2618176129096886, -1.305165336734632, -0.2913231469208346, 0.051642055643137355, 0.11620539536154703, -0.11059145070029129, 0.009441709350717567, -0.1553112816562681, -0.27463114300172703, 0.06295667495907335, -0.6426876890851715, -0.07350296015181045, -0.21436195743867, 0.1260927268833685, 0.07807161094124804, -0.1573090766866893, -0.48813937406802305, 0.09989066120351464, -0.4176117426617354, -0.16941913862628288, -0.33497372034508316, -0.13346161024393038, -0.10446581600369341, -0.2758614416615932, 0.038611899551566034, -0.1150447564604113, -0.07337975472195327, -0.30714515650964097, 0.025720603762544827, -0.34920789187003165, -0.20244073481916228, 0.01999087613761935, -0.41759004211576084, -0.19298524404186743, -0.27703324643980565, -0.20894922290066728, 0.6747879918631641, -0.6788530735745133, 0.35288336541654214, -0.064326055765467, 0.026016097684447313, -0.12167124117188914, -0.7337290707952239, 0.06869210455578711, -0.0680237150166834, -0.2201933726526521, -0.10301202487058947, 0.10868999713733132, -0.12394369753590805, -0.808142924256221, 1.3953642344357273, -0.10597881674547943, -0.17125812239787316, -0.10170910902199783, -0.1886093780302855, -0.23448437369362998, -0.02013674579120297, 0.03843782303099099, -0.42636995136964856, 0.26945452994812824, 0.008399112517831726, -0.04925660666788189, 0.5815249174815469, -0.4692835381290847, -0.09285461333332418, -0.17699151549786496, -0.02642724241784803, -0.275545499856536, -0.0035965972570156306, -0.33419143020630226, -0.1880797942843037, -0.003222911048108909, -0.3352367792882618, -0.3898952039653605, -0.19939786446388952, -0.2556151918471163, -0.3897764740611305, -0.04428010080087469, 0.33979746868200056, -0.040915643528395024, -0.37694483126316375, -0.22617046781030695, 0.08331483942973113, -0.07045167407150621, -0.04306511792178271, -0.2748536949895718, -0.14377953818790834, 0.47860556538949667, -0.3572130853393506, 0.6510935974390384, -0.05883989118823503, -0.08858605786468757, -0.33901600328570825, 0.2306883905605939, 0.5210114702779677, -0.2120483029419524, -0.10143630497526848, -0.5277287347747692, -0.08575666437974436, -0.13204214934784797, -0.3232602172718051, -0.9094054977494782, -0.03457416186585427, -0.06534336889788575, -0.016216411192958375, -0.06293645428406233, -1.0778228547420508, 0.13168742939605838, -0.11189333530687351, 0.3273276283409765, 0.07834229260018528, -0.28442515309724203, -0.08438417544168128, 0.40967983484798864, 0.6688668188403674, -0.46335877455653557, -0.10028902157119331, -0.2187291620893171, 0.07162418431782731, 0.053840452414558995, -0.8274680745217535, 0.2675112061009316, 0.1100535522875711, -0.021075350423764048, -0.029025364027233814, 0.10142079495361607, -0.23064197082023422, 0.04324743670934593, 0.020318915245260697, -0.08638885271977821, -0.11661318931029786, 0.039632918534086785, 0.018794803935364805, -0.022742286604462966, 0.08026864301997241, -0.07009791538290719, -0.08658279749168861, -0.22167199810573482, -0.16943512021556179, -0.12433496084487668, -0.15255509786520144, -0.009927424858802308, 0.06803619191021687, 0.3712882299217773, -0.017198731734821636, -0.032571755028088116, -0.14457742276255492, 0.053584982836131084, -0.3265543055342042, -0.10303989838686603, 1.2563995175955731, -0.3967530272679573, 0.04062772651170382, -0.19438622404891312, -0.03574947090465021, -0.15456845262659452, 0.03707419875575593, -0.5071574698689896, 0.03459087032595611, -0.1799517231250309, -0.17020711486582368, 0.02257916694556377, 0.17264833234656166, -0.02763388341118462, -0.14324968419176293, -0.2509518728350522, -0.1590079818909045, -0.4733654406594797, -0.0010845551524016751, -1.162424372134566, -0.3893503942811259, -0.008430277958463808, -0.024058752191509544, -0.3231115700976102, -0.050086956849405985, -0.28873707658778575, -0.488532658210322, 0.06550619729203475, -0.14211198100654754, -0.3812429742995796, -0.015218372506205821, -0.11110785243536775, -0.17348083932225675, -0.308912293252371, -0.09441026830772435, 0.04167794474769855, -0.05814054711177735, 0.26969680385476175, -0.8688414165187605, 7.964170849964966e-05, 0.09232625570642021, -0.638674927029704, -0.2460123957720776],
[-0.4594425270803383, 0.31995413145879115, -0.36106824234380064, -0.6650725997295723, -0.14080934677846793, -0.1892607781506785, 0.888711704246603, -0.04623067405344522, -0.03762133048305558, -0.11399851740278774, -0.11598825944892367, -0.050840167800195596, -0.030286864389050365, 0.0066276626124645, -0.5596117495551178, 0.008130200298908484, 0.2714629737356772, -0.01129321394585767, -0.39949295849711264, 0.23961653593783333, -0.44008579135641546, -0.37909995213348063, -0.030425312294865187, 0.06882510772055768, -1.3666598679769248, -0.4167267001672003, -0.09041163847274811, -0.05500232315261671, -0.04149706408259002, 0.36059748900677185, -0.23090601236138483, -0.5131889761541224, -0.37752172536594, -0.6316078812153741, 0.08521822120636027, -0.09478171453474761, -0.36123033140527927, -1.1935953210252832, -0.6458480674145762, -0.2766556099811266, -0.5786335536632475, 0.14106004241949266, -0.5738044415130801, -0.00989566231104155, -0.49026809366432855, -0.26817555952720296, 0.14613926478696548, -0.23624182958684434, -0.5136418199882479, 0.8284461827815321, -0.7318095251356982, -0.04278521286263568, -0.4861783886320532, -0.09770699926677665, -0.033072741195272176, 1.6260175980235634, 0.026078219421831618, -0.19085423821022754, -0.17754767401357294, 1.4585441708007343, -0.028696178075610524, -0.33928963951833135, -0.026039990148717638, -0.2116446379328076, 0.007941886722972694, -0.3806158182312793, -0.3407323374058646, 0.286281653674159, -0.0790089626043442, 0.008172792827208139, 0.013548677988231088, -0.09644055168349738, -0.7417780263533463, -0.1384684328184214, -1.5440112242296287, -0.23688948050954448, -0.4418469419300022, -0.1305189896599501, -0.3943064914169652, 0.20559373228550187, -0.4379957518056098, -0.026106290962373154, -0.45805826203346606, -0.20750260151406077, -0.11619402622352147, -0.16001699892052443, -0.6574215491941343, 0.017873437608172903, 0.7617351057086834, -0.3059121019338754, -0.3482195265853287, -0.20302393304370334, -0.26556152502040464, -0.19029562537804376, -0.4654929164853496, -0.02603366014870424, -0.10407232783509213, -0.04855295690903644, -0.05701257975807812, -0.21995974092933027, -0.23518926800251597, -0.1974116537342427, -1.9998705995825794, 0.09132284351824048, -0.21354258959797012, -0.33387948303397486, 0.046388327276173154, 0.09975372656977069, -0.15483834827668158, -0.1827879320314317, -0.006596146134706416, -0.09876057096317277, -0.014380594740953604, 0.12022938350488786, -1.1254353320040136, -0.29974563006720634, -0.026226054528031152, -0.6457040957750008, 0.08100893574035593, -0.5743382229333958, 0.1176159801762298, -0.16636431682678665, -0.11510314462850169, -0.164067937745372, 0.12522973563351264, -0.11792767792477936, -0.22608082311029695, 0.01757914180456241, -0.4812917232399073, -0.2337148366396883, -0.037478141308090185, -0.5196185976874521, -0.29217372721701185, 0.053319215132326375, -1.141268888686715, -0.03322154491714943, -0.20697672504185657, -0.3958057907220392, -0.053476458816687125, -0.14663091447786034, -0.10769728467025108, -0.18177459689135259, -0.17754983728221707, 0.10209558614011395, 0.028790082702154555, -0.36972094530663907, -0.23458975770770008, -0.15291969112391982, -0.048809224968643386, -0.35343698150021985, -0.044084092648306324, -0.49811443128991373, -0.03281885368778697, 0.0866032976014424, -0.9575645670003476, -0.30716366421284347, -0.19518506799927018, 0.13515613268426627, -0.21379152382033026, -0.13814144904601136, -0.2467541446737609, -0.2771819467159248, -0.4589500291176241, -0.07348873108058705, -0.2906853099151577, -0.25248368133733484, -0.13724207552476536, 0.10502314146983419, -0.07077729752426903, -0.18995393674948216, -0.29324975046601126, -0.1977030543878581, -0.13727476287916357, -0.0773088818862081, -0.0012179602387532044, -0.14399301989923913, -0.12104920586293598, 1.091149347745409, 0.2682472460793185, -0.005378596557166024, -0.01783118553294569, 0.008197755891290901, -0.9393158276411163, 0.03297305180781741, -0.1249628199910089, 0.2022651530181282, -0.1761740072086948, 0.08978401389432304, -0.1522833305854194, -0.176050204604255, -0.43625867688873377, -0.06564423801272519, -0.3079353823278352, 0.019232390821900053, 0.1952449564669851, 0.04703565986643773, -0.28766968792229525, -0.18509346612044758, 0.0036964025093391595, -0.02542132363187963, -0.5672101300863536, -0.35938001809160175, -0.19478697757288949, -0.1528198663859868, -0.39732699529075566, -0.5310171924058159, -0.08719243673500804, -0.00960252178069566, -0.14601892168733852, -0.3836821171280304, -0.16309605209703154, -0.9413477119292124, -0.406664767558367, -0.37971051132160394, 0.5542929229972737, -0.19382685449108142, -0.12548623974349848, -0.646152172570716, 0.23600116561248974, -0.3340633448611681, -0.2669861971847471, -0.10168353851437961, -0.7392275209450107, -0.09828508578784251, -0.10607602484295742, -1.105667080488086, -0.207826882483805, 0.08697563688333643, -0.2691687183523531, -0.22718612376750108, 0.10784148101088546, -0.32481921051756807, 0.14210880227813313, 0.00025273039130284646, 0.004321691255133961, -0.2620845823624458, -0.19936991393298073, -0.13103499133058882, -0.5153803857928704, -0.23699697142567136, 0.0014334543110050733, 1.6026350105215033, -0.3837567724868719, -0.12188474919297855, -0.27837319501298696, -0.261348184168035, -0.05087103859210437, 0.05299784528255303, -0.38173644838913473, -0.2774580705311128, 0.1192283566819994, -0.2740372133934344, -0.2899534210996488, -0.05388149056477077, 0.321520280298162, -0.017966861318873328, -0.3752330219706698, -0.22478754122344435, 0.4932468111824017, -0.40806281108762754, -0.23516894376069256, 0.25416410773942305, -0.11443719470261343, -0.16480176552362902, -0.08172983797936885, 0.06524286147766804, 0.13524400819580934, 0.09171418684354188, -0.47587753444774705, -0.5695584367953982, 0.18676255060146885, -3.3804409054469886e-05, -0.20582287313470968, -0.0938642136014176, -0.30302949914430477, 0.057572897388453004, 0.06851725658359507, -0.21060127553294153, -0.09595662759084261, -0.34851148971592477, -0.05470413822333618, -0.4436263106271601, -2.0092486062933235, -0.025876706116630926, 0.6927899316255833, 0.17518616321709166, -0.19081555322574772, -0.15819112273056277, -0.012537688342990808, -0.19965869795211943, -0.10945055339169879, 0.188797303877827, -0.40736955187751184, -0.22954025564831398, 0.11264908515432301, -0.06782572448833148, -0.19838728444057416, -0.17677762007244927, 0.0033540471246925013, -0.19343912151064152, -0.04337496490888218, -0.10838933387145204, 0.01860974236737298, -0.00715542222671501, 0.14726081314594183, 0.34467706020962163, -0.22475489266940804, 0.01762450209090875, 0.18210871397560593, 0.13267369052680092, 0.3416593379210137, -0.38820538092646945, -0.12859033566710434, 0.38948874522641547, -0.6365124420357715, -0.2561464871584942, 0.03477943523235871, 0.015841299026579665, -0.37042121404018497, -0.279029949851916, -0.18184102678822983, -0.4976072441135149, -0.11543782227324782, 0.1010958676639246, -0.04029506396077536, -0.04834674263338012, -0.6413164443549927, -0.036736490872135194, -0.39491386088922725, 0.05690584968382279, -0.29738149379586526, 0.021025036201285704, -0.8841523308136212, -0.10592859623289032, -0.18552124594296815, -0.040950577615823054, -0.28913326937031547, -0.7035794300581838, -0.08091686506854728, -0.012740540072978791, -0.5952823916909055, 0.12020024103765183, -0.3575857386920368, -0.41142518993490657, -0.08620265524118378, -0.04764797976235172, 0.25788928898908603, -0.3652566389291493, -2.1770566909923583, 0.406949207140763, -0.42225798358342914, -0.014263732904481486, -0.33425770620856426, -0.22747574350698285, 0.08536639312577783, -0.2733315143679832, -0.039623267214963794, -0.39982489139575234, -0.025507173106344518, -0.05625472167043017, 0.015174378709001426, 0.25486129002277463, -0.03804733231773147, -0.1809782054182348, -0.016932445046482496, -0.06555616909265903, -0.8012016528513765, -0.2509224241171903, -0.6330578939769984, 0.5553417563865587, 0.36407390417016644, 0.022742793608440427, -1.4962421836934183, -0.08159303337673879, -0.18512499298020607, -0.08406076085403291, -0.08091495202664374, 0.33569814092794986, 0.028152113573131406, -0.07860232273690745, -0.14638587910188877, -0.05042383833809917, -0.009779838098825908, 0.24367077897150904, 0.08597308310542212, -0.022408600019450182, -0.292691596422064, -0.11211077775593829, -0.03075301441473298, -0.3507929288337512, -0.4216638413577932, -0.12112915484694131, -0.11460310124474457, 0.07658934568749923, -0.46679060245134085, -0.07549039209335186, -0.052762563433161894, -0.19712997762143186, -0.1705916660918726, -0.21667623259651322, -0.41193215400902844, 0.005137438570118746, -0.6106799512566831, -0.14540043779474116, -0.31436371397137863, 0.3971159526538152, -0.18903334026135257, -0.4932045124475227, -1.3669324437812531, -0.17233009174103353, -0.2518074570915371, 0.03759348963242307, -0.2114636333816059, 0.12522839555592385, -0.09676614222724134, -0.535113768935523, -0.06328570263290588, -1.360343448619798, -0.04061575324743876, -0.21612976396723138, 0.12195395755209937, -0.10411660739622626, -0.022690984944069122, 0.21993238724439443, -0.14605374788045636, -0.148533989043835, -0.13784151674971498, -0.466459163004408, -0.12426396671049714, -0.16004398402395806, -0.21657437224678147, -0.1824948993771177, 0.245883766500627, -0.07916057071979136, -0.1719952884866422, -0.13084397188382665, -0.21942524107069147, -0.11394210422030364, 0.005808007636606404, -0.498947248498412, -0.08818808816606698, -0.1820200015053947, -0.3029435890464624, -0.3707205136623154, -0.4137394582085582, -0.2806474368373404, -0.27833419138376264, -0.20233980484776343, -0.3533710440952817, -0.37018053498748366, -0.18237049956314347, -0.4248862644402157, -0.2956303641973214, -0.19698956404859877, 0.0046608076390236355, -0.3605163967210686, -1.105351848760218, -0.1603792372448822, -0.16632077020866506, -0.3408796862588326, -0.1658983186298331, -0.2848711741351566, -0.2493420384335769, -0.10805236057448285, 0.11500573635025177, -0.23652648951567148, -0.04941364203178111, -0.026540364867541504, -0.14675291133024046, -0.12933743026257827, 0.25582912794492324, 0.008868765406887797, -0.005167520735597225, -0.34739464298392125, -0.3414973526567528, 0.19329471945331725, -0.2239512631188848, -0.1634530742765776, 0.7365735476330649, -0.3462817789916873, -0.7634584473185103, -0.2619071988274218, -0.39409159138170663, 0.11616578341936724, 0.051701673766856006, -1.3484041222546723, -0.06426627880293798, -0.383062739930725, -0.06853727053376385, 0.25126982320980684, -0.5104521507945261, 0.04748920855630983, -0.3379849142161194, -0.25756568064297575, -0.25237503941936273, -0.32494919275104023, 0.11268752191718079, 0.12170779639841438, 0.1349529076623418, -0.3279479451294823, -0.08113594776314034, -0.41442610858490886, -0.18169343339971752, 0.013973704337343816, -0.47569791981526865, -0.19478268655605538, -0.6663042814227598, -0.184633768253836, -0.8177139235868935, 0.08112054527706726, -0.20237311093156865, -0.2865676245581756, 0.07811471618066756, -0.24974672453708746, 0.07942472656813286, 0.15909101994942812, -0.030224441414548654, -0.0069635431511241306, -0.3578426080222563, -0.04240649318141394, -0.14713349943887977, 0.12076115080568768, -0.36535136925636963, -0.19448881143554866, -0.18295332427027247, -0.010897180186062939, 0.07955019720283221, 0.6800040069059129, 0.08535225494983101, 0.035027506254427125, 0.0479620294485618, 0.01800570925119954, -0.08520956797056094, -0.2659024840083112, -0.12662391908028722, -0.0034845729728606735, -0.005660312315319656, -0.16628374625822784, -0.018476813014407664, 0.043343402937450384, 0.29384726834221714, 0.06556137039643663, -0.2263941636870881, -0.22673038046046323, 0.0176020187976944, -0.20381870360276366, -0.2818291823914537, -0.3506067765314549, -0.3543204630900943, -0.42428353775422545, 0.04772358924048659, 0.19704991516476048, -0.16530202120759221, -0.445636714358895, -0.015459877163395702, -0.3729525751562916, -0.0436610186713828, -0.6896831609096961, -0.047426108091987246, -0.3231984891180492, 0.018281855062210077, -0.10292201462370969, -0.07460748147460639, 0.01791254512435548, 1.837516790409551, -0.018766649716984652, -0.38392324780450804, -0.14504111913160112, -0.09890625728377264, -0.5262180480349816, -0.12917338914603158, 0.046256495458697686, -0.3451131056012393, -0.2991314081378442, -0.5700173776440929, 0.013862230675819246, -2.3060940729062605, -0.12544941741422044, -0.023829748300874996, 0.2820056583913622, -0.1740107638836584, 0.0019848998326361346, -0.34817813052408275, -0.26480626512788025, -0.03180281478918746, -0.14488321117226338, -0.041090343144897505, -0.1262019341182863, -0.11443738966108892, -0.32559187576826715, -0.1502570181468293, -0.21278828919388149, 0.07832428794312975, -0.021664316582985725, -0.061077788050711714, -0.009462412122416764, -0.18195194137549506, 0.08790858148504495, -0.6557636201481858, -0.34262859793193823],
[-0.24921155905660416, 0.23476179373468908, 0.0015106550647994682, -0.4080087337145137, -0.021460414265768054, -0.13619126685605332, -0.34369685404092987, -0.01662517954755542, 0.026167640945613087, -0.09739882051940091, -0.1429808921773512, 0.04216910544215523, -0.6026726966765424, 0.004168321935056445, -0.1983821343851424, 0.027099602228071135, -0.38877827274289484, 0.10492178127579702, -0.17122049835480996, -0.02642780858571861, -0.2951727978597298, -0.1664401377321196, -0.0502404365669096, -0.043409287785750546, -1.2137434404755483, 0.3137164067813827, 0.12215701224207307, 0.027690307182001283, 0.02810173120991296, -0.12639693154554718, -0.19916104125830517, -0.869911214561349, -0.4065114480450804, 0.7900338517815045, -0.07173087853714044, 0.09835586408344288, -0.26676545626213505, -1.190599082416213, -0.17624292977617292, 0.012725934883907523, -0.6362383201086182, 0.03401596304784358, -0.5488971720555411, 0.01942386349817266, -0.08459271499569045, -0.1974988175317129, 0.2503242369120138, -0.3507613751657567, 0.2891515180731828, 0.5430727723133022, -0.315695239365284, 0.023174677865145033, -0.2675248180088809, -0.2381472735179811, -0.011363469429087985, -0.1533249133893259, -0.07080026213207723, -0.07466067840902768, -0.05164016537525773, 1.3226100173843107, 0.02602937717866417, -0.3661275988338978, 0.38265276343880483, -0.06915822180446939, -0.008055584309887159, 0.20786494909895495, 0.09453288084541601, 0.2604165279034626, 0.1827341027811381, 0.037847480668372575, 0.09755819048973015, -0.13841023744683265, -0.7253356853806484, -0.010785834953323902, -0.9940430020934332, 0.08570522724989643, 0.1474723314863657, 0.39302026135783774, 0.0256004326111037, 0.1034562792962465, -0.47550152761783604, 0.4805190781552507, -0.137666421277179, -0.4400169807699576, -0.18506529420225382, -0.19912543199557395, -0.434011682878323, 0.07612152227986867, -0.540529316111307, 0.07564302276163969, -0.1584133241009817, -0.11709503637640277, 0.004798119185160639, -0.13521034211231567, -0.061233991779819585, 0.29569188544196573, 0.009333812080277823, 0.12494905757022963, -1.043811755286849, -0.7845530073477903, -0.04211880040666905, -0.030993242125558172, -1.6338971263567896, -0.010729168878941566, 0.3772272795236965, -0.31151309510553404, 0.10074236766307793, -0.06566191608554203, -0.1656519852495099, -0.06849545548873583, 0.023396419593324228, -0.3547782915789159, -0.04610594539846653, -0.12478566750727794, 0.23239078273369643, -0.28848115704667904, 0.011305715045842291, -0.7272028713613414, 0.11908859623269609, -0.2039726096679715, 0.016413134873196313, 0.08775522005575895, -0.14291352464597185, -0.0321968589144657, 0.0190143296908182, -0.10555320270878268, -0.14669593064974193, 0.02524448975007352, -0.5310314595658853, 0.0027937384481041263, 0.02833151359786047, -0.45454830361359105, 0.1971007490945292, 0.20776518342042072, 0.31745295529059114, -0.028087321998616546, 0.02016862981680601, -0.03810203153533776, 0.0492211510154784, -0.03357881115284473, -0.07150914567237228, -0.2766479135106578, -0.10823187406518878, 0.10468385376479887, -0.05664547377468355, -0.1356483910333172, -0.18901505865490573, -0.2085781491237593, -0.07601453299892869, -0.2402809255351535, 0.1170309250653653, -0.4910166767386517, -0.07180142028959594, 0.026536456178517413, -0.4202762846634526, 0.004102833341180628, -0.26351499503204207, 0.010097735839789377, -0.18356474244218385, 0.1280515371468887, -0.09456923608530884, -0.18007968130711202, 0.05657174456480864, -0.22947114776534888, -0.10089884808788095, -0.06424644876444605, -0.10733866379370748, 0.05730518865844386, -0.09580901732397491, -0.024060627072243893, -0.19044170938383984, 0.03402219640155132, -0.854500945264535, 0.060629880083836726, -0.21562505016145636, -0.3609100415506333, 0.0028585901065646896, -0.954970171975972, 0.10399139226766366, 0.044154006546097045, 0.1260838335092852, 0.16020672754498508, 0.37566996296951205, 0.17430063841245266, 0.017854301649802156, -0.14581406929484653, 0.15466787524022077, 0.25284010367751575, -0.01767592114720331, -0.15333673249073, -0.20514449033400836, -0.15269836017124894, -0.2439946254692137, -0.02212157502009254, 0.05460302849338756, -0.27813673577489945, 0.04472611754328804, -0.24645838672607331, -0.09553130572674778, 0.06367768039459744, -0.35934386461832046, 0.21787982005930434, -0.015686747466881375, 0.1087745937801388, -0.12775935314751388, -0.3416497196588184, -0.26680183481730146, -0.05418789249476359, 0.023492106265700113, 0.11088398049323261, -0.10513475827852606, -0.9413303417846866, -0.25770355612254986, 0.3244414483195954, -0.20415071335668716, 0.7052018239314041, 0.22078891199832457, -0.45892063373922903, -0.7926302016363227, -0.05866199383061784, -0.3803938662837602, 0.17904829895928234, -0.2804402626979506, -0.05318751168070993, -0.1324978760634558, 1.0225752720604566, -0.20902843864845602, 0.07069070798379717, -0.10867512131623894, -0.037856299625975254, 0.17593954500152753, -0.08341201161707815, 0.07087423554838689, 0.02074254467458696, 0.15603537607261375, 0.43537685513135604, -0.2994358814268288, -0.1097518642592353, -0.04925405758388851, -0.24032044889062684, 0.02363127666236656, 1.6554495236282196, -0.2104501637817474, -0.30621294970032215, 0.10381529726272919, -0.0062831569834869636, -0.014632594709993719, -0.4004929400299712, -0.584383140142684, -0.12191034612403982, 0.43780952779626464, -0.41248464424888376, 0.03498676670185043, -0.016492886353714622, 0.000835377655256244, 0.0881067077670621, 0.06602562969630453, 0.03250029512125642, -0.08799602906335506, -0.13741531987473668, 0.11821454100233708, 0.7104155516278745, -0.015922972054542742, -0.11548087153314801, -0.05769495284394544, -0.013434390056756292, 0.47207254006226834, 0.03487226972615604, -0.38073345155784066, -0.1878380114133364, 0.10619449953399981, 0.07886326973256953, -0.1865508473365867, -0.13942097882785243, 0.1342187703588062, 0.05147240800018464, 0.02019505606947141, -0.053660613966343015, -0.05550974666561372, -0.22105075670183613, 0.0731620931028874, -0.37597818643695435, -1.6100044521009125, 0.13025338865928224, -1.0677459807256489, 0.2726493542401491, -0.3283391639027884, -0.05226487516826426, 0.16067228230126973, -0.21745174228700556, 0.06041658666724181, 0.3459766078858854, -0.23255855375319867, -0.1849387380713363, 0.16730282371733077, 0.059383470587648104, -0.07908831939866832, -0.15665708619674695, -0.12331383012342204, -0.02196364346805574, -0.10940563600211732, -0.21033242951376474, -0.005001277444560683, 0.10838933923823071, 0.16954970427188687, -0.48825623815134234, -0.10791221169720848, -0.20715071045906167, -0.008125514788926097, 0.19074056647431736, 0.5193274074516857, -0.45528252019160986, -0.20073778536606623, -0.4214096200305452, -0.6815472968898596, -0.1862847355672827, 0.10267041860276407, -0.06063703432493816, -0.059347147312515856, -0.28836732379565977, 0.14585937238297703, -0.3244544966562737, -0.08448288216914364, -0.09414556037566316, -0.03997246249135523, -0.10938453523882473, -0.2682127580496751, 0.025876496004247056, -0.7514075719761819, 0.2569352996708282, -0.25781124693124435, -0.05678691721599264, -0.4705080876227766, -0.1251219556115939, 0.12942586393991334, 0.08874534961655262, -0.0336320270414815, -0.254965687454626, -0.03402858328426127, 0.005814912698180073, -0.25042203070514, -0.2844071174530222, -0.08539986897444235, -0.3573179665005781, -0.045185197008064344, -0.7704973496083193, 0.17479601070971912, -0.3585855081504213, 0.3900347719158567, -0.10556280177616909, -0.4497156654701398, -0.002126271970333376, -0.3518306390098053, -0.28221804880587503, 0.12234397853469269, 0.06635568120650809, -0.13054993366408726, -0.280729598315385, -0.048264761135141235, -0.811524947049777, 0.00858482435572046, -0.23006903502126488, -0.03225443713605295, -0.05537397667218843, -0.012827903997804318, 0.13881614932090636, -0.29062469350130976, -0.6083262304495561, 0.46190625416817305, 0.18712715060463045, -0.07902690408148615, -0.12883940444289207, -1.7485566053791959, 0.0785686109416269, -0.5650656537562306, -0.01578610454595547, 0.008042935279690163, 0.49429038648757107, 0.06398725819223838, -0.10509212923381281, 0.001223318552253907, -0.07452033172485156, 0.026280361679592794, -0.20666697408558168, -0.009535440250902312, 0.07676036265411769, -0.4087784021534768, -0.07916455379303042, 0.07048809224306703, -0.18979303451204435, 0.7624129545160961, -0.15600392100674493, -0.1881982174179912, 0.22566246146679936, 0.1332338006614424, 0.18067665462732846, -0.1482584900337927, 0.08018202780207413, 0.013610349810921189, -0.18553477245406907, -0.25129334774459183, -0.3016967821709841, -0.1496912967490575, 0.01227744904410453, -0.010070115440460926, 0.510662894624466, 0.02133798068929381, -0.1566842570458303, -1.4308330414106039, 1.0144777393250985, -0.07824090034212643, -0.01348772140847698, -0.10974388403151769, 0.13700183142255737, -0.051249356116478, -0.643239537524817, 0.09049908307511369, -1.4920518721977496, -0.03272007419387742, -0.07067370840252916, 0.12039871465105303, -0.18588856998060013, -0.13621622743966605, -0.03157347710616112, 0.034307742268968475, -0.0452431663204174, -0.06080372970724986, -0.11532816475854862, 0.028996220784188707, -0.08842480996992136, -0.40400178619213395, 0.0718542135667154, 0.04948882929542555, -0.12956819802394265, -0.05445698727181012, -0.2710918237112777, -0.23894321040824085, -0.010683575803345065, 0.07138632464617167, 0.046779077013021725, -0.14265825322108067, 0.10621615226880016, -0.20978035032548242, -0.588374227927185, 0.5753516719445083, 0.0950775763543578, -0.3672865854141862, 0.03394216595828036, -0.059859834126529864, 1.5959412360390244, -0.0699712847636341, -0.2350720232562711, -0.22075281188016868, 0.026986929778607067, 0.10638311165637808, -0.11499644133747977, 2.6903008666479247, -1.1980631408375022, -0.06590856096190581, -0.26265932296358707, -0.12624468299902508, -0.017687827484736605, -0.15408276636876458, 0.1341396219633815, 0.07808242521582218, -0.29683242290122197, 0.12999846438213966, 0.23395731643490345, -0.03811270175047297, -0.7587946662605338, 0.45840835617337156, 0.02201997226364338, -0.060552357610857324, -0.7150691665759689, -0.296509473628105, 0.10444049159654784, 0.08784309301263897, -0.09958608821628631, -0.9993136260048506, -0.2855063273799181, 0.7741834159388952, -0.1704137124177398, -0.07696575037720799, -0.3475156418302989, 0.03645969081264637, 0.19635313750308295, -0.12555076510875457, -0.2247092924669927, -0.18367097868052776, 0.1822500473659236, -0.305211414208983, -0.13175643613522217, -0.26226255914724494, 0.020211958147063346, -0.18525957648462846, -0.2873520123035117, -1.3154936966125494, 0.24370897697040533, -0.23061160960381413, -0.3863541201252628, 0.042331084279961566, -0.6475296858999369, -0.34991320014297495, -0.05458688738374088, -0.46749164514753183, -0.03183888016831826, -0.6248168714777099, 0.017883394968428116, -0.13205742332576517, 0.05148408823916829, -0.06588420826809119, 0.09740040278996585, 0.14679174898047312, 0.023270877473462088, -0.10482292854580452, 0.1393043795572718, -1.0613656511291627, 0.10614385086818792, -0.1972175089993254, -0.13048044439286086, -0.10367532570274439, -0.6368817898795336, -0.5519457478889706, -0.14803614230722004, -0.12878921141181718, -0.17477319403236619, 0.05620426484760402, 0.30682219838777414, 0.08161593900001465, 0.13062895380194228, 0.0567388209551584, -0.031215237217958252, 0.13348611590509535, 0.018410603959177763, 0.12945928507310644, 0.045680133655989726, 0.019420361805460164, -0.09265593652897541, 0.01070842826607293, 0.2003277885654503, 0.03237075310368842, 0.016410558051218768, 0.2722375356777289, -0.20913232338478038, 0.018776702523926922, -0.13267506947802984, -0.4668982877339704, -0.14068393217940628, -0.24380026270415528, -0.2176680349611798, 0.02529800979849034, 0.22620859969724127, -0.09605787023764613, 0.03830851076229085, 0.07699880204306278, -0.06127914398577762, -0.019515313661304528, -0.5001654908320289, -0.13950164957366007, 0.500267503778354, -0.13325640331646932, 0.2574523869695002, 0.016594015240618324, 0.14435413588660678, 0.2903692600253033, 0.1390031239701538, -0.11937161872504093, -0.7854497112314444, 0.14715436540590626, -0.2099813049684579, 0.010795668972038078, 0.0568923592635786, -0.2137113213534083, 0.008522030048631429, -0.5236401293473938, 0.04758207338693618, -1.8231833410780673, -0.3066105979084205, 0.1754608833743067, -0.5785317755556834, -0.14842003863526393, -0.18103809803659118, -0.2884697230709816, -0.2984418753012034, -0.06048425206829965, 0.09463421072969473, -0.3033403995824402, -0.16452165844762037, -0.11068656174724932, -0.10422999017821913, -0.13771076251616365, 0.26932393174970815, 0.1355484160009328, -0.0013868327570657737, 0.14806507637262034, 0.016232981023625793, -0.10467094503210914, 0.0820763917217752, -0.7912693207590208, -0.5230448117874195]]

def sigmoid(x):
    return 1 / (1 + e ** -x)

def identify(wih, who, input_vector):
    input_vector = array(input_vector, ndmin=2).T

    output_vector = dot(wih, input_vector)
    output_vector = sigmoid(output_vector)
    
    output_vector = dot(who, output_vector)
    output_vector = sigmoid(output_vector)

    return output_vector

def cnn(board):
  result = identify(wih, who, board)
  result = reshape(result, 7)
  result = argsort(result)[::-1]
  return result

def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return copy(new_board)

def find_scores(board, scores, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores

def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        return index[1]
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        return index[1]
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        return index[1]
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        return index[1]
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        return index[1]
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        return index[1]
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        return index[1]
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        return index[1]
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        return index[1]
  
  return -1


scores_player = [2,2,2,3,3,3,4,4,4,6,6] # 11 possibilities
scores_opponent = [-2,-2,-2,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    
def test_agent_5(observation, configuration):
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    scores = list(0 for j in range(7))
    player = observation.mark
    opponent = player % 2 + 1
    moves = possible_moves(board[:])
    
    # Only Move
    if len(moves) == 1:
      return moves[0][1]

    # Offensive and Defensive
    our_choice = win_check(board, player)
    if our_choice > -1:
      return our_choice
    else:
      our_choice = win_check(board, opponent)
      if our_choice > -1:
        return our_choice
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      if len(moves) > 1 and win_check(next_state(copy(board), i, player), opponent) > -1:
        moves.remove(i)
    
    # Last resort
    if len(moves) == 1:
      return moves[0][1]
    
    # Scoring Moves : 2-step Look ahead
    value_opponent = list(list(-10 for i in range(7)) for i in range(7))
    for index in moves:
      next_board = next_state(copy(board), index, player)
      if win_check(next_board, opponent) > -1:
        value_opponent[index[1]] = list(-10 for i in range(7))
      else:
        next_moves = possible_moves(next_board[:])
        value_opponent[index[1]] = find_scores(next_board, list(0 for j in range(7)), opponent, scores_opponent)
    
    final_value = list(sum(value_opponent[i]) for i in range(7))
    best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
    if len(best_moves) == 1:
    	return best_moves[0]
    else:
      cnn_moves = cnn(observation.board)
      for i in cnn_moves:
      	if i in best_moves:
      		return int(i)


In [3]:
#@title ***3-step Lookahead CNN*** - test_agent_6
from numpy import reshape, argsort, array, dot, e, shape, argmax, add, copy
from random import choice

wih = [[3.6920327192794615, -7.244057919842115, 4.233188964719989, 1.8043895415697038, 4.58275418359571, 3.0131849314057613, -0.16333164174715767, 1.503592951719824, -0.3740112461577557, 0.6063470379383881, -0.34058090813975045, 0.7268186530088414, 1.5658032501071046, 0.18987026349696098, 0.8046410738404773, -2.1380941332634924, 0.4458324185420854, 0.2537056973463159, -0.37465373092949333, 0.2581530173361692, 1.5680692392766982, -0.4100501138774406, -1.2876112663665622, 0.3239536985421265, 0.7780142197046687, -0.7719841932889516, -0.9447582119185367, 0.8995173952001375, -2.5284837441189607, 3.263793927397769, -0.757714515981968, -2.1050619497503136, -1.934786996437022, 0.4019735245550619, -1.173217057140023, -3.4384540793731104, -1.995033685709947, -3.8777044508161347, -2.852919811855892, -3.828481323615157, -3.3978379286326827, -2.668429701524594],
[0.26252528770641265, 0.08537400836603515, -0.10583574069138049, 0.26011879115402947, -0.16486155895725668, -0.32940383007750407, -0.39519567370767267, 0.012206109224897545, 0.18182075606524437, -0.07279158019367832, 0.07328013001363921, -0.020362456654926697, -0.889062271933661, -0.12553931425021, -0.15094403181432214, 0.09463315970201755, -0.6180914196503561, 1.2973410578338371, 0.2634412634772544, -0.12734773977264705, -0.2432061585832564, -0.009503360153626503, -0.36897325660628777, -0.5772582206660327, 0.733478272818461, 0.918168214528241, -0.5959710404484699, -0.35578815471694364, -0.6462851028879573, 1.069803686906628, -1.193053500911227, 1.0538317422891812, 0.41675342642891283, 0.7987953063510606, -0.3967371364903907, -0.46069080177891564, 0.36976799663146753, -1.7898386478968409, 0.1941470776331503, -0.5243041669212902, 0.6889454127318864, -0.8405579260938107],
[0.17752335014114676, 0.17974957996128155, -0.24931626340358432, -0.025959167377366622, -0.051093346564727686, 0.11601250092831167, 0.1729329314280967, -0.1490529131384966, 0.010170475089422546, -0.4544952056244338, -0.12196302108374676, -0.35512680137277636, -0.252463015066149, -0.059282670561936465, -0.5593973928240582, -0.38538941081096484, -0.7810787006207537, -0.2669608572773698, -0.5599957604657106, -0.06556342596819804, -0.3154079032746093, -0.002365838187085241, -0.26871324187568907, -0.9302188240611856, -0.6281326149823561, -0.6454672599450898, -0.4641170537249279, -0.25781368212188854, -0.5206421421497552, -1.0145484985207556, -0.6844176553770029, -0.08592805942340521, -0.6094882877835573, -0.7245178858541936, -0.8829521215674083, -2.463312468966375, -1.8274421722587624, -1.9978042615841989, -2.400400688071654, -2.2677830503363667, -1.6197895116289702, -2.0418136055728837],
[0.17925667061942868, -0.01574420965574411, 0.06828259303294873, -0.2183115233295555, -0.15204684786836226, -0.024672366174690003, -0.23757204877973492, 0.1503300689499923, -0.3496383952777234, 0.09487247903016591, -0.100380375791682, -0.14218674789283706, 0.024159892576882898, -0.2901808258295947, 0.37597826390366457, -0.2481520187971826, -0.14628782343165112, -0.2184532151376845, -0.2387585238917683, -0.1792384366229478, 0.011970137604445277, -0.09338497844811665, -0.15587011029095715, -0.5552241895385753, -0.33999018975924855, -0.401410874861212, -0.19631039734074476, -0.2262417656931879, -0.5839808202631326, -0.4786145838917975, -0.7172293722185812, -0.011984660661584915, -0.4263857610641744, -0.5845990734661946, -0.7835368296633993, -2.152675459189594, -1.8579732548073975, -2.060097328605771, -2.4045872548962333, -1.7443484987376132, -1.8116698856974087, -1.7906034252160008],
[-0.1981038730883346, -0.18276408685198015, -0.101808368204481, -0.12814079829956299, 0.10282372801747129, -0.044741337106355285, 0.040513965596121984, -0.19212601158809034, 0.06263110153940937, -0.42909670588867654, 0.12229452008009878, 0.09525428146128437, -0.12685268179097578, -0.08130523576379052, -0.01824211403821459, -0.48274825604280863, -0.24091002654990296, -0.06859233679534774, -0.04382087351652544, -0.27022140902796327, -0.04632327360369071, -0.33374006867085015, 0.31722552422364175, 0.13268107058694706, -0.3720490998354789, -0.26696889174336713, -0.11731463822251012, -0.1445592899112771, -0.7564575790864035, 0.3738989652374938, -0.582733072318517, -0.6092280482428568, -0.43695680879132043, -0.6671391214801992, -0.6458436481859038, -1.3018984186872413, -0.6513257206555594, -2.213320042175003, -1.986012210254286, -2.1467673062611725, -1.4021256590108693, -1.9634111721606389],
[-0.008314895052833936, -0.1787864717648438, -0.004949101981369209, -0.07521845142297474, 0.1468092884774125, -0.05333205274831611, -0.08620343780492468, 0.01863809495472172, -0.2619596483469773, -0.32279764236116826, -0.39187485274770417, -0.024864117564070125, -0.2176997384164171, -0.2177538476483432, -0.03411048584896557, -0.42164237171789737, 0.1646653717230308, -0.3013835283676457, -0.22511994858407292, -0.39921890944782734, -0.1642482569353005, -0.2762367052926482, -0.3194013997143978, -0.34413314616230195, -0.44772617931140896, -0.25483752920537134, -0.3657057031888192, -0.20722087976951023, -0.4752167471932325, -0.3614797647049482, -0.41526314686527205, -0.22939055539084685, -0.47702117402014227, -0.5891719769332915, -0.6145286856470878, -1.7903368757171108, -1.3227052175203946, -1.5910696869092136, -1.9825816132484555, -1.7005408635317198, -1.5647571816103063, -1.6827372107923628],
[2.5362053056966802, 2.4058538466356363, -1.8181980115978533, -1.0157103963511027, 1.5936430485040112, -3.936557008038382, 1.8024356045166516, -0.7373541241903451, 1.980090043395433, 3.498851332743481, 1.3691659531887401, -0.10865582215602089, -3.418876272738125, 0.5446006594556551, 0.4521303692465155, -0.03338818031827306, 1.0080689692022002, -4.727554549069765, -0.5156566571338314, -2.1572721003562134, 1.4420987262814025, 0.9748571160859978, -3.05372985110104, -4.718452359605314, 0.19316299912331644, -1.702190212913726, 1.123337919304037, 0.22954277542855542, -0.4421971235612093, -2.167123844818533, 0.576713787704928, 0.8941009797717341, -3.81236792407585, 1.7330114176193652, 0.5619854804220318, -3.257597570831327, -2.6120988942491885, -1.6448220564376639, 0.6111504170383164, -8.267751871894802, 1.7753651210442558, -1.6073162220773556],
[0.10629962930100126, -0.1444316769421932, 0.015433074992264635, -0.19649451237422771, -0.038733065348265094, -0.1025840501784094, -0.14395070860067505, -0.33595686344777187, -0.11988828777952147, -0.006998484423895029, -0.3280407711450199, -0.09832493694391696, -0.21498088269613552, -0.15481907942274448, -0.3352401180475094, -0.08631333038265747, 0.3090922953670953, -0.6248769770569869, -0.2648724027533989, -0.6475531962633709, -0.32721885169776926, -0.7081243445251159, -0.1750386775792655, -0.2509338344376779, -0.08314461299210942, -0.34583589823076855, -0.2869768172368614, -0.07406201117762692, -0.5981335488832001, -0.5262505215165822, -0.09352948155850556, 0.3502001598696306, -0.5003216674595267, -0.7005547507303147, -0.43260704504695674, -1.3843603552984234, -0.9085833121975441, -0.9026092935343676, -2.069755274857403, -1.2678624966624832, -0.741924325748288, -1.001383606648032],
[0.20255095607572876, 0.09610014667650126, -0.13941477010009215, 0.06394629371160314, -0.11357773634769032, -0.15686829111494088, -0.14276753407110504, 0.14420290944312766, -0.037956468615229125, -0.05915506090059916, -0.24433405301838312, -0.09762655818333123, -0.11970848930679874, -0.09270336364035013, -0.05193078290897494, -0.34080132582382955, -0.16379329673909937, -0.29890473923375405, -0.07226437126656088, 0.01720309805878413, -0.12270505720935286, -0.2869931523411432, -0.19017300848645918, -0.22894278675309745, -0.31421105590953585, -0.43581393219712355, -0.34307887882781324, -0.2909387677143875, -0.7199026488963939, -0.5840416374101399, -0.37653834856992846, -0.4664741500404451, -0.2759214712924747, -0.37690363597558374, -0.5173357374085191, -2.1972930859247044, -1.7168601222289785, -1.8544342331963277, -2.361383691620973, -1.9078100896061192, -1.7151042531785987, -1.7640395115521332],
[0.16792873408204545, -0.06038443187205756, -0.338743958478707, 0.09682311091252631, -0.09912042096047202, -0.04271029010441317, 0.14728147216054902, 0.09581963684632398, -0.09842098776509466, -0.3083211244736272, -0.11543249129359406, -0.23657850251573573, -0.22345331932141976, 0.05236406503712367, -0.03793592924029281, -0.22810600539249423, -0.2525186799762783, -0.00565562365294716, -0.2911155452407524, -0.16902613040168293, -0.13505478142525135, -0.3581840706471435, -0.4215346001717991, -0.21486887580441316, -0.15794442826370442, -0.4049605029415886, -0.4645872902837102, -0.40353362485836014, -0.8059720195899872, -0.37799042838133284, -0.5320031069574915, -0.3728961493472302, -0.46960957005256115, -0.49501917290027136, -0.748000077250521, -2.615289011142299, -2.227891403374709, -2.619514076325971, -2.410939143652046, -2.6099943927613825, -2.5101007993783293, -2.5122513480140887],
[-0.09087546551475865, 0.05469744443480035, 0.12360202951739449, -0.18239335947383725, -0.11698378641567339, -0.004934753783868976, -0.10282407249710168, -0.2756392412854901, -0.15791156700856884, 0.15942375869255276, 0.058601374908208936, -0.24955503807497975, 0.06928370515826561, 0.06770423294257914, -0.08379989757888545, -0.17573311461380864, 0.09061406510463524, -0.20587226219210344, -0.011780579194252435, -0.12381337714579015, -0.07940250729607617, -0.1347910083959297, -0.24979394365684124, -0.1694182854456895, -0.16593746313980268, -0.37363872143035726, -0.35273437523558443, -0.2592432306422306, -0.6911707351606581, -0.5080072957974445, -0.486972971328111, -0.3922435449523617, -0.4725027810958101, -0.5299996892112904, -0.744856274411746, -2.4584310948515067, -2.1182584386161736, -2.4906094313928215, -2.738555289763694, -2.399103752238234, -2.2729993678137084, -2.4477905681851215],
[-0.06066769951479112, -0.1298750812173647, -0.24598169191018965, -0.1004097037734866, -0.0828247361362373, 0.05010280529763124, -0.11276752861333186, 0.046336950169430596, -0.3513899868744168, -0.4244017394796529, -0.012985182370380948, -0.060248235580467054, -0.2673682410754724, 0.058572703520683826, 0.04742473719868934, -0.23743655830538127, -0.5261690984275181, 0.5296103072280637, -0.2500337706436513, -0.21362677879784236, -0.22572841911923686, -0.4291359926752966, -1.0556583331469374, -0.5108879273694872, -0.03951805681872705, -0.46632825815438594, -0.8029218406170845, -0.24845764013502383, -0.07069327533195527, -1.0748051190370136, 0.6953451783092637, 0.23613223590354687, -0.39941417518918076, -0.5987686534495663, -0.6625687630555238, 0.35698015438843284, -0.19226042751765166, -0.7691126834077132, 0.9162034845677918, -1.8209754256299175, -0.8390794397512208, -1.9648395936663794],
[0.5055894990696509, -0.30700102589094025, -0.756403287453546, -0.46108111060852064, -0.768482674333231, -0.4319157409931807, 0.31417344707235323, 0.14714314130618372, -0.4901511865914631, -0.5883066839026314, -0.3881320300039488, -0.32475916001974797, -0.649317956914093, 0.32146885152546245, -0.26724739820728866, -0.8105442991363302, -0.32458539982348095, 0.15042976239768321, -0.060699331871550355, -0.6349176628094743, -0.14200927263136426, -0.5081750524698221, -0.8169995220258063, 0.2772471123372893, -0.023332239093072363, 0.3005263755465825, -1.139798737475903, -0.9929749468016954, -0.40451483630836527, -1.9600180319077003, -0.008693765513740605, -0.548576446229067, 1.6402680350371224, 0.14093131089406813, -0.5243594180516029, -1.9841581853368961, -1.3667499420791176, -0.11189112458368686, -1.434819540921151, -1.620170219834856, -0.4256793528846523, 2.481070934375057],
[0.08332091698088577, 0.0872268027014748, 0.12728588011944555, -0.003436077251087078, -0.04234222333007598, -0.10989101809393775, -0.04816431561231924, -0.35195099992295137, -0.02782549618992084, -0.049105212372008614, -0.09462746294121745, -0.07281555747243268, -0.0906549378856271, -0.08698630636150206, -0.10552682549797025, -0.2987359850220972, 0.013342621646104615, -0.34642563374093477, 0.0201023898026383, -0.07057568694415192, -0.33431192184141645, -0.38148061601911215, -0.18648681225310118, -0.19279436690186016, -0.672824183512421, -0.1194710099670246, -0.011493916463980203, -0.18145235780090113, -0.7812251156867673, -0.5403945733363654, -0.7572436016072082, 0.11628387380883305, -0.45326912127547125, -0.6227998733255093, -0.10223563918757411, -2.0177882968679586, -1.2061818754390334, -1.7546858373639092, -2.0544623656678906, -1.2055290692199174, -1.3094742127356958, -1.1443517494676776],
[-0.06546975549232886, -0.3907193681093094, 0.27207839061110695, -0.6879538570022286, 0.13565851331195056, 0.12491738123465047, 0.12815428199238577, -0.12245147628329284, -0.06706176994281718, -0.23248786060401505, -0.5061190990112547, 0.12650589826288453, -0.04526539459099698, -0.2542709093211703, -0.06809903914652814, -0.4526570217473938, 0.4147214289416033, -1.227106533677836, 0.5364105016338285, -0.571662415894246, -0.28729016271267954, -0.3645965025766386, -0.016953417592223564, -0.30278252993971366, -0.6170208120548321, -0.15184499081282188, -0.14244641512355644, -0.5263144461656848, -0.7764201874991881, -0.8124612144798187, -0.24607581385064375, 0.040450595813110966, -0.8579359923620178, -1.0406247970995313, -0.64411938408346, -2.8750633238408456, -2.402422553310893, -2.678357135173968, -2.7973381140235545, -2.6308405782975455, -2.340604699392349, -3.0984768075139204],
[-0.17345382458779637, 0.43452663398412283, -0.20703775228575713, -0.1737200662964857, -0.31228273761922615, 0.11109628587686933, 0.3454727531564346, 0.009835515394316341, 0.20489588047098303, -0.0064692317631280085, -0.2404051874532601, -0.359331921435166, -0.09172890900516716, 0.2365967283438156, -0.14371989880372715, -0.1658852440046037, -0.24313937056358728, -0.15863378466185094, -0.1006307006783972, -0.16842824850001445, 0.030465955126264505, -0.22353075962763164, 0.06988137384012487, -0.22639242551384903, -0.2037779109471093, -0.15771556368554637, -0.185146032235658, -0.34769563983283025, -0.24127187366053818, -0.29694471208181455, -0.08522613982607996, -0.30506225287160765, -0.16156239838430003, -0.5863162695926124, -0.4678016134409775, -0.31553991856320085, -0.7267620623169766, -1.0640065687309224, -1.397221818004542, -1.111524583347383, -0.9134042856143889, -1.0043944745502058],
[-1.143395701175637, -1.2061107124116015, -1.348396939877448, 2.4586851143050112, -1.4535431519949684, 0.12964971843445772, 2.9177204835833446, -0.7125432431642184, -3.61001502217337, -3.758178102675495, 1.4156624454764772, -2.4595456948545933, -1.5667351764971864, 0.484146870009338, -2.5582705144622366, -1.44825379854444, -1.8217580763029353, 4.806030929621572, -2.581881724078662, -0.6325121940873261, 1.589620057392546, 0.24656881271644698, -1.0714380014607243, -1.2550241872222472, 0.7478170619939987, -0.5618398355345191, -0.6593406778431484, -0.28930136248451477, -0.27953114512100674, -1.0968096461065457, -0.4262013285809544, 1.7694291290139703, -1.3425340770060676, -0.2481958092585689, -0.1384247171410796, -0.18962815466696392, -0.46839675558503835, -1.8348223419671412, 0.540257602558185, -1.051753791502515, -0.8756868417709948, -2.030109018856068],
[0.1955612625737871, -0.627335214799185, 0.06509641554235851, -0.13761491252278593, 0.2874811089593937, 0.007748028911928659, 0.2340252038135999, 0.030796332550254355, -0.29379093308240517, -0.4383505186384552, 0.13146894974680054, 0.22788864980970225, -0.04349479758651049, 0.09779460394030515, -0.06503219174848968, -0.39421121076624116, -0.3138488361462421, -0.14601431286188665, 0.585163327329306, -0.39424056469987445, 0.07624540448097343, 0.09166609452497171, -0.427818286801801, 0.2251761678850975, -0.41817788575203035, 0.22268103672170636, -0.15941113295516407, -0.23648135036800458, 0.7029239217941758, -0.602913896253851, -0.5189254223078476, -0.34407500910977007, -0.3592456108571896, -0.5750456281810952, -0.35351798603435275, -0.4130187838379706, -0.7136166748822395, -0.7772844677756322, -1.5508614556899587, -0.5379382487751556, -0.2471568284370135, -0.9718577434443137],
[0.08537190412027047, -0.6423703967299358, -1.1090571892413572, -0.32362512632946655, 0.3385214348356562, 0.6939336415150053, 1.0755461728437596, -0.023726122706929222, -0.4718159787320922, -1.2176880078748087, 0.2822189738761779, 0.2268741425836726, 0.3725982356184618, 1.1474727237307119, -0.26986363145280573, -0.18520598633210889, -0.5205597448445656, -0.7235831630242093, 0.0844246780175919, 0.28292647112583336, 0.3784691482311491, 0.28634285626801265, -0.1747884990864307, -1.0175410049286457, -0.1645818059894885, -0.8473030001146239, -0.5736156453980109, -0.12958266023535225, 0.4885825510752945, -0.8164930692609704, 0.0926691459088164, 0.35585505529718486, -1.2922177608009833, -1.00807551439433, -0.7420278944732019, -1.5362948042765205, -1.502605848756868, -1.5813000658641438, -1.4002071829733447, -2.6922450768703863, -1.8004008415580681, -2.581058172521485],
[0.44771867718504027, -0.3714198202648779, 0.46911486962924326, 0.3794118183098849, 0.3545569412389165, -0.0536025191484126, -0.06546881803716177, -0.0128424228508469, -0.29585764383546537, 0.49694956706539656, -0.11384439911968758, -0.03397497969722869, 0.9560005597921628, 0.26854512957635623, 0.364621435722468, 0.3823365124975977, -0.22477798618118341, 0.1580634238798518, -0.39917775043960024, 0.1770609212556175, 0.2696061257769195, -0.10489933591032814, -0.6829499325091896, 0.07305194040235707, -0.4124723437552714, -0.589857672024159, 0.2839524347718052, 0.0868781107877945, -1.2023938417751234, 0.9280017433719672, -0.010405635328648799, -0.5555348593079876, -0.8157263603377649, 0.21338746369164913, 0.29609775226193924, -0.5518317553359047, 0.6846176976723256, -1.0179603788455545, -0.7177073059826781, -1.8748698588137913, -0.3426283364443814, -0.5483981032499297],
[-0.015371088175939055, -0.12465011771785577, 0.04518214814753919, -0.05043108441036863, -0.1928277782200663, -0.055716902119186396, 0.07265622743044356, -0.16523280431223747, -0.0806570044697366, -0.10409452185243415, -0.25756021552452746, -0.21201827388747863, -0.034367084639593454, -0.07548061577034165, 0.08205615032027552, -0.01777489109343277, -0.13586946726371982, -0.27262993699940785, -0.12142553987971033, -0.354523857647453, -0.4096550919171272, -0.3602836897081539, 0.009076017378329238, -0.2834459108323818, -0.524514268794333, -0.2857012360026934, -0.3851566028248068, -0.0928745446293783, -0.7573602484066627, -0.5282376138718411, -0.6032470361831146, -0.4691225665913047, -0.44923757771755873, -0.33292894419484936, -0.535930471396437, -1.5498866644593308, -1.2484236098833208, -1.7149282007692512, -1.702089466197689, -1.5277986856267272, -1.0687324770407154, -1.802178882372064],
[0.024865342073357333, 0.02054422326917201, 0.09695076691084502, 0.21557118074299364, -0.044670640962989776, -0.1389714288281299, 0.10238334005837002, -0.06611702394405143, -0.12854947921522825, -0.17928148990511092, -0.21853198692784168, -0.12452340768442001, -0.12536235179791416, 0.0876308908154015, -0.0388218463742762, -0.15253733052620624, -0.36499141250508604, -0.0900164324107219, -0.11181282094640689, -0.3135793246327136, -0.07041062535960804, -0.241415705323314, -0.3431018249129747, -0.3429072785412178, -0.5243956893558539, -0.3623773757924299, -0.43088987782414206, -0.2180727502427593, -0.3273694829913016, -0.5842092789260798, -0.34297286182759346, -0.3520949136782254, -0.4049144415236024, -0.5632498777153272, -0.6879351839829798, -1.3677282865212246, -1.139696815504914, -1.3540406180532945, -1.5080565363770755, -1.519727303972291, -1.3134867902801892, -1.5498399140614092],
[0.14526480283891074, -0.0545285042752372, 0.33476004407516735, 0.39121750219433377, -0.03374065660797898, -0.15086061782683113, 0.08401231677457517, 0.13505271590171467, -0.10610265607558521, -0.03530155699791448, 0.17445479277582085, -0.039637680130585724, -0.17321843146913316, -0.02452277665105239, -0.11571902450963373, -0.1476801245366144, 0.04684245463174988, 0.10254757682103316, -0.29809332248719217, -0.2269962802807524, -0.23809209150620927, -0.2975284119817178, -0.4601781540574663, -0.3348046811727048, -0.2008712800625889, -0.35310871571594343, -0.43738926350302476, -0.2906310517031112, -0.633966925498942, -0.1817492109700602, -0.4659721449823532, -0.5542465659322846, -0.3945500485959715, -0.1795164754422282, -0.6034059386338934, -2.095394773238216, -1.6828568063385527, -1.799354478432041, -2.098023649494526, -2.020419189715587, -1.9982237590800647, -1.8574066367462636],
[-0.07382605126806092, -0.04231728235136375, -0.4034123441635899, -0.3748994241944943, 0.26250974961741497, -0.051103139634580706, 0.16107589145650852, -0.25268138644544946, -0.25476639614755003, -0.41654326444171685, -0.29006072268770694, -0.019456951339422664, -0.13906696070742924, 0.043268600478024714, -0.27221807783667124, -0.312225736676904, -0.1360364305831533, -0.19427256217088418, 0.09701763262743991, -0.25351437712977243, 0.03505218474616996, -0.019908312761693976, -0.2235854732706309, -0.17103136007301184, -0.48046718972801605, 0.02241819527007838, -0.24653337628878355, -0.13531925518460225, 0.012435209102060323, -0.724208700479322, -0.010598623424916248, -0.19897089771493226, -0.44016548513654624, -0.7375505949482217, -0.4168059086979254, -0.9330959787613171, -0.6021130755031383, -0.6566090349277873, -1.4964420356174593, -0.7063313164647517, -0.7334281184824122, -1.684058740184331],
[-1.75515832187717, 1.4607749567054271, 0.6517634396938232, 0.0007548893903647639, 0.5884411205564143, 1.1978417812827449, 2.334430968226296, -0.8535606223467875, -0.31801752924653826, 0.14313490615516997, -0.12635857790896124, -0.39928478086937147, 0.37226249018709684, 0.9292034019824799, -0.09206644045084436, -0.4704926506279224, -0.2685098390607096, -1.1734440276515954, -0.25899290353100224, -0.14336746539549058, 0.0009584618327996825, 0.47025959507747545, -0.28266556562166395, -0.4139951045226136, -1.2318398299441984, -0.7760085200833701, 0.2602021014044231, 0.04201860269129431, -0.6639844217031596, -0.6600948847204916, -0.9530909311550319, -0.5641144345703417, -0.9206332411592372, -1.2829997806135174, -1.0183524112627536, -4.217927674975928, -3.768064434585963, -4.074108360794878, -4.11467978044772, -4.039310610319494, -3.837985181721694, -4.008745130388019],
[0.02095957068619997, -0.06013766389203641, 0.33855144339524595, 0.06746475457534072, -0.28390446248065543, 0.11198933712507751, -0.5250313137234294, 0.16828935735977524, -0.0616756559864506, 0.16376251769548344, 0.48558136933037743, -0.8525525521315596, 0.5210525540355149, -1.464070860155038, -0.28606320308732436, -0.44353326745351834, -0.5884456773572461, 0.2411331834117998, -1.0592303651359016, -0.3417764289744204, -0.6969479512272191, -0.49138934544420976, -0.37960221520306175, -0.6617968790031586, -0.6632473524007726, -1.003192117364226, 0.4738572656009205, -0.5625562911903564, -0.5453094598880031, -0.28841511796515185, -0.43213997862582065, -0.7976434847211041, -1.6681409949998738, -0.1557215943448605, -0.9379913672523753, -2.745347997472898, -2.220686580189934, -2.9204080043934098, -2.1520279112171714, -2.5598162899801142, -1.802439326787199, -1.8593454032755496],
[-0.3521449567287281, -0.23945086809360908, 0.02741914739017275, -0.251123975330998, 0.5230684531293781, 0.014490110194461158, -0.370922908882794, -0.036099035421116935, 0.1620267545245951, -0.08663208247094681, -0.11106390068879697, 0.3114432297296608, 0.1583791864694019, -0.10001946895640287, -0.3632966413653924, -0.0521011279212678, -0.15750964063562314, -0.18003809411839444, 0.18941660810524982, -0.14550511574631406, -0.18908707051497417, -0.3292811739508378, -0.05992498994100506, -0.22452248237488184, -0.30425117865918466, -0.09880191504110777, -0.1309924450600799, -0.19009210374107452, -0.4317167561662477, -0.3553848154933186, -0.3779045477532461, 0.09953627041804713, -0.6123303720057726, -0.49912042355329184, -0.5624532423037639, -0.9268580791527947, -1.1121951135026311, -0.3814834372757434, -2.1360403318960683, -1.4492178918616971, -0.6620822764893015, -1.5765230125200143],
[-0.058452664651324426, -0.06470287705246502, -0.14500732256983312, -0.25203973379085337, 0.0377808459026233, -0.033096668916650494, -0.01600914468323933, -0.17740246729034506, -0.1493399784319572, -0.11724157092101692, -0.3001076008076963, -0.007452093562371415, 0.009213862764731547, 0.05413269870414449, -0.2548182523903182, -0.27222023927260014, 0.04711195064481913, -0.3650399419167339, -0.09007486622547352, -0.27029331518228095, -0.24464949951176998, -0.2547098033870838, -0.18983986621093413, -0.15883793052282882, -0.3855552123223721, -0.22641471633831342, -0.038235991217483035, -0.2553845117145108, -0.4491347248760267, -0.507600041445003, -0.45256026614582207, -0.08785266284982027, -0.574317502396582, -0.6879974912582304, -0.3829740823235428, -1.2527341214805312, -0.788519083063699, -1.0696881480302955, -1.9700140847381358, -1.2255502581523656, -0.8859451250397217, -1.4339130570130403],
[-0.09604633521115238, 0.07556333754586184, 0.02207325099336286, 0.27140052524859676, 0.15879929085346212, 0.0067243573131562815, -0.004674832200451748, -0.10180777740912714, 0.20933407387891614, -0.2810968273675339, -0.013742915480500759, 0.22411602547307546, 0.19858486039290468, -0.2735326270603641, -0.10710930697488259, 0.0781196765466029, -0.24591223555328268, -0.35051472881836815, -0.2826199316129046, -0.37653723266803, -0.08033300275472294, -0.16548142059771287, -0.05517794666386909, -0.25076083574209845, -0.3724063339167498, -0.2756193386949852, -0.42454613464266816, -0.13248261983920592, -0.6829170838047162, -0.4511992712687185, -0.7164550920923495, -0.4237104315584464, -0.5140318692362021, -0.5848917648895928, -0.607086714975448, -2.1433229929060373, -1.7446820093480981, -2.3157901732325943, -2.154250541224422, -1.891707214206854, -1.9410082773876183, -2.1414308642694992],
[0.3732770138392405, 15.725916430971449, -1.198253715063106, 1.3678090880129037, -0.7269535598699441, -1.1748401699597355, 0.8305713646323264, -0.5602520688262627, 2.8008438005650707, 0.43134886499960107, 0.6175867013661918, -0.3836507166369061, 1.1346919367867192, -0.36822179822879797, -0.16022244507112768, 1.5912434980002768, 2.3324483230263096, 1.0722894697998664, -0.7113127280011864, 0.7544581126680184, 0.21741831288043162, 2.2783402981733776, -2.6220065700070836, 1.3431867368613724, -1.5580729901463792, 0.9625833855093122, -0.4649196263127378, -0.7303026045340282, 1.8282694445611154, -5.619058962530935, -0.2692745413368545, 2.7141130007349377, 0.2509530715196994, 1.6619472491603355, -0.7193813130678247, 1.9312468769123379, -6.075203021661724, -0.3894153224466484, 0.13416215856577018, -0.4807091598633493, -0.8982054083086259, -0.47186445904337404],
[0.22503227928026165, -0.24074426339228466, -0.4225001837757535, -0.1790317441130616, -0.11196946956697203, -0.2600322015406641, 0.26737749556162127, 0.18082021139902613, -0.23212019229119987, -0.4899457916437121, -0.6576515604924711, -0.27485391433189105, -0.6594351883861932, 0.10538818723702531, -0.24200396950803924, -0.3166459089310846, -0.3543764907633365, -0.6589840224706225, 0.17278565011285854, -0.18671905949091266, -0.4081611705907309, -0.585408624748397, -0.8166428748886858, -0.3487518672096678, -0.60577755548229, -0.44589342950535193, -0.8267335803838133, -0.5727911197291582, -0.610058377190458, -0.7713783179898838, -0.4415380105594589, -0.7213551161620184, -0.5064500144314658, -0.19697477898959626, -0.7706436171101071, -2.8922709113850815, -2.150837378332374, -2.460622062993499, -2.5867951602051886, -2.2007472086422837, -2.2442194269819127, -2.92327029069987],
[-0.04429878564186331, -0.24856846685892806, 0.029189542910507224, -0.09505424684282979, 0.2589476653193234, 0.15418211109699964, 0.031660181662875604, 0.08355240187710455, -0.15130700593372978, -0.23298241698692973, -0.39440554840260106, -0.0010438013332447242, 0.2400223010135582, 0.22992035801474747, 0.09988398228803264, -0.5687275056216726, 0.8396224854462386, -0.6776361516607324, -0.04841730919979992, -0.7442596209884685, -0.2179492332963332, -0.31952227177642567, -1.6257988983865026, 0.6296302310172879, -0.9667038347792328, -0.4943949354328362, -1.1377190755160334, 0.12137025882080751, -0.847822440639472, -0.5291676526082237, -0.5403200475400807, -0.8249635985834419, -0.6620131894252423, -1.1424814331482787, -0.8020916691204416, -3.373333294422899, -3.022187842389267, -3.2098095593273057, -3.738119301022201, -3.222525771826186, -3.145795544807479, -3.266611408124505],
[-4.673494924052404, 6.794208801551214, 2.683780103563824, 1.8026162342306116, 2.5954798580192713, -2.018286184666132, -1.1035631820772622, -2.19710621136744, 0.7094566761460299, -0.6615989555572181, 0.20505708853211968, 0.24847003813658022, -0.17421256870904703, -1.736010069328295, -3.2176158963531125, 1.3313985072573145, -1.3243944658689766, -1.3717453233822121, -0.7560151434936526, -1.5754092630668666, -0.6088327744314229, 0.09404827564585946, -1.4264528571413726, 1.2935676991406109, -1.1546720911463038, -1.3056453638256245, -0.6053612243741266, 0.6725238587528437, 3.9176717869764537, -3.92417928309709, -1.4883809403621502, -0.23457603570849475, -1.531801590073301, -0.840489482167839, -0.5202774084026006, -0.9641449204026912, -2.828891457398795, -2.6393709391281486, -3.0509868051574163, -2.4926436544568693, -2.7501089469846773, -2.673878454515785],
[-2.714511345601054, 1.209365792193305, 4.563325481733012, 0.19400833456913355, 0.7963742606456992, 2.532766515987228, -2.4271746153180302, -1.13186617051102, -0.5214757042141731, 1.8063446310456095, -0.8046495625562383, -0.46225339661179304, 0.3327071761919275, -1.0022108901115325, -0.4746267630867156, 0.3416602880410878, 0.445487879625353, -0.25182485267794713, 0.5624038230038151, -0.2860814597456586, -0.2517566961650685, -0.31595813580681087, -0.4085595361947453, -0.43909332508934407, 0.20207818543650893, 0.18447619802636134, -1.9261972370235263, 0.04622703101354826, -0.6603921374894519, -1.9344443224026502, -1.764642595287342, -0.02807681270974771, -0.6750221417465033, -3.0336364176439385, -0.5900909088022418, -1.823302516491533, -1.8501049194483434, -3.007128139123376, -3.671506985644053, -2.7610397015932233, -2.7875112854182835, -2.402156495778638],
[0.17111202951604618, -0.3088619480217017, 0.011025806285391642, 0.43608480201203864, -0.008880997686901024, -0.19208354582845194, 0.1531557764502304, -0.025867444391290045, -0.327260761383204, -0.07435012718069413, 0.1284990472515695, -0.008268392294796737, -0.09357243192723126, 0.050123683385050445, -0.027987541424582475, 0.007193424844867323, -0.18391473633066258, 0.09299629849803008, -0.3480208392066269, -0.23156482070877757, -0.1570792856121801, -0.4052215105016971, -0.407240609056124, -0.24841208803521403, -0.1876061728048541, -0.32897408859666044, -0.6531665307261345, -0.2905185956707563, -0.4807341347991371, -0.3345229580008008, -0.2815320045195842, -0.553203330621534, -0.33398651440901267, -0.19223200890265985, -0.5511468818568344, -1.5732909355630154, -1.226980701918296, -1.3739019849725884, -0.8500795492314659, -1.580057092212803, -1.7052852433755914, -1.7237453973227101],
[0.6166354036649603, -0.20137885184431978, -0.36462983925621767, 0.2551141569238575, 0.4526214494232443, -0.10028316007216626, -0.2687577611822337, 0.46954543372549074, -0.20361165180233734, -0.5488749939161031, 0.16156909411103895, 0.1176301748167048, -0.23342166519194177, -0.23285411204716286, 0.07112408111815635, -0.18898944957992467, -0.3182024453750378, -0.006469318154071129, 0.0204398798699284, -0.007761249573599897, -0.3374187212934838, -0.2161513309309552, -0.5042250537062684, -0.29821724731333277, -0.43613550512460575, -0.4333788342492742, -0.33886822478346484, -0.12762611988176115, -0.5525466120373692, -0.5319906245846342, -0.0601137955562959, -0.5818120828906945, -0.4814186734851383, -0.39446661732255056, -0.6941002922761589, -2.3610469472691293, -1.935722638156045, -1.7546218661672313, -1.657466808454079, -2.3334992720021392, -1.9420448485437127, -2.201011188359682],
[-0.9762968325498078, -1.6063909136335834, -1.2019377267306344, -1.2405971050842686, -0.5372658683548337, 0.2982819592011095, 0.24003319709192786, -0.719336495775959, 0.009659677830491011, -2.1934225299653334, -0.6246609895646127, -0.34299103908437567, -0.11853352916143492, 0.05495914482788877, 0.0359454498146215, 0.5407176956395262, -0.722503609549143, 1.489540040142214, -0.5161053605723227, -0.26182306790543863, -0.8525232341154113, -0.545921106694942, 2.2683120082976935, 1.9014552353464043, -1.4358023213568278, -0.10131698937343539, -0.9717108521477628, 0.09207869677039132, -0.5997895525682756, 0.9523388617274412, -1.3543657877080355, 0.49098851487607026, 0.4685487212972064, -1.707363884107214, -1.5611816265853826, -0.48358718315135724, -0.5156173341221606, -0.8508213341050632, 0.05991468523188276, -3.778965365871524, -3.002779000331931, -2.188161980118241],
[-0.5590236704463857, 0.5217432118693973, 1.6293663493930146, -0.4983735990316814, -0.6116063879454214, -0.06412878477905408, -0.145740273436359, -0.3218319204307991, -0.17943185480133061, -0.5287244919578767, -1.0738059389725367, -0.10955894147351855, 0.0909551129987969, -0.0694472750916585, -0.4505531688952534, -0.0848614097599648, -0.8926386831428016, -0.7209932353171468, -0.34021398013170423, -0.6037672718672534, -0.19787589167761935, -0.48695985012557186, -0.6675465649914781, -0.9264703570042757, -0.29141294899637177, -0.4986880906973021, -0.125151523750763, 0.04069220082267271, -0.9160892060145012, -0.49020056983376875, -0.8063090200090193, -0.04520778206277622, -0.059036741484398615, -0.4127349008174294, -1.0022163425951744, -4.299347622791439, -3.9639844079724673, -4.374861467131016, -4.3666437402331795, -4.443047546142418, -4.180996047247751, -4.362314432096318],
[0.16237937810425115, -0.15132190995228803, 0.2815395099489342, 0.3318700981899957, 0.33118333059423316, -0.19427146996072117, -0.20690407225404261, 0.15607695226338136, -0.3085612603737431, 0.05570284412164603, -0.18633491297754276, 0.1522223607856752, -0.3133626320501628, -0.09677008236105718, -0.18145746613842742, -0.7053609810813791, -0.03509869450673757, -0.0948131362092502, -0.22468221972810126, -0.15069326044563733, 0.16544469910850537, -0.27794551284561464, -0.9686530759913473, -0.35626136494749855, -0.6648625338031277, 0.05368878779608988, -0.7717665590674172, -0.20792367173706316, -0.9749392963009431, -0.6463834849456902, -0.5049043824477635, -0.5666023749872561, -0.4410056650707676, -0.4135104453120718, -0.7713323649110813, -2.8061320120462923, -2.3257280271043563, -2.789856164161096, -2.737163092951002, -2.7912592878400315, -2.3953005393219744, -2.622385644344352],
[-0.6991470619590491, -0.025991827056992798, -0.2687994046144002, -0.1476264683210056, 0.11746801773000895, 1.030406685865074, -0.2579224140504001, -0.5384251476134242, -0.0936398687119135, -0.37750818852247153, 0.4095850472776146, -0.2642246290688643, 0.5280817691954084, -0.03824609672316731, -0.20988087187787785, 1.306673950453972, 0.0004111058651099008, 0.8156371100678008, -0.3591246648396943, 0.3749285683490961, -0.10157460171042404, -0.5878431294258537, 0.5664523221787827, -0.19559725042630077, 1.30018677224547, -0.8073297824168382, -0.2763098867803805, -0.2631838964658713, -2.6390075858773665, -0.6080101760443694, -1.0090442367839398, -1.1413485799255458, -0.3957675434919107, -0.7719353048640994, -0.8627569212894448, 1.1561952119869856, -1.891078101352692, 0.24424282829491428, -1.0188569433206354, -3.019459196631514, -2.449042756951432, -1.1698248798524198],
[-2.241220405744418, -1.020268369007875, -3.9132402716063748, 0.5536255803074618, 3.230342934051394, 5.214325636763689, 7.806600324427212, 0.2765902806035271, -1.0479020768046579, -0.7707013807589843, 1.279834968955318, 0.9275136141789557, 2.358677940912391, 3.5384113087801548, -0.45335968202908794, -1.1227320169656343, 0.6609102334141145, 1.1920034871095329, 2.027319675167618, 1.5247979466750992, 1.5870128428069241, 1.9560280149323883, 0.3632174995155504, 0.40732131898068874, -0.2722466974628522, -0.7998209430338471, 1.6994052830516002, -0.42326061077795446, 2.924048284100336, -0.34863836907810847, 2.6254689412290584, -0.25878003362738394, -0.6293486461596839, -1.9705385129595376, -1.2765803392970854, 2.6134856255642287, -2.2510524175727005, -0.8507961617586878, -4.9330087728011165, -3.2126982559767896, -2.5396194282340905, -1.8576508888307777],
[-0.019807527642475276, -0.11415793691933154, -0.11181333578219611, 0.11325756614322026, -0.0009727909808175498, -0.12647821712487198, 0.23630470220080227, -0.11023400792659477, -0.22371577689900604, -0.10724164886728617, -0.0853548255928459, 0.07180669236114859, -0.15571119711685297, 0.22143417860655967, 0.002529021549161668, -0.07331834640457928, 0.261046517919615, 0.0020788171620835035, 0.18165080606905382, -0.05582255053468485, -0.1551027832264299, -0.22994028785695014, -0.3260220884321532, 0.1276231393590519, -0.46448463937777723, -0.16240876812109564, -0.42874781025304687, -0.11584186106947225, -1.013399115707399, -0.04879589802864124, 0.03869979301571086, -0.6120497697485182, -0.4822986228562003, -0.2449431151007201, -0.37957846739802936, 0.11233615260477464, 0.16271809663083356, -0.5263610089375462, -1.152552373057229, -0.9413813575434141, -1.4802139712076385, -1.643756295690793],
[0.09929797586224985, -0.32112221814152925, 0.11668432297016869, 0.39762159662813445, -0.318612949624187, 0.0875329353160645, 0.2518911563355575, 0.21135040254014986, -0.4053666178090404, -0.16667457429884855, -0.01893308630079109, -0.3344752943650251, -0.027885279891543296, 0.38522514194746077, -0.06112852271071178, -0.392227822929314, -0.10820076212057868, -0.03958385410242583, 0.00594031526154504, -0.2439588971465956, -0.011351325676570722, -0.5620762472139194, -0.3782677720259842, -0.17992099534399136, -0.10790491363976651, -0.5634744116422125, -0.16822828444740412, -0.358510916093425, -0.3320543813751501, -0.3756440738148832, -0.25681423644096735, -0.3851213217051314, 0.013403298278493967, -0.6185145575667276, -0.5861892909381727, -1.5887493887928248, -0.8776151509604702, -0.6778147008354061, -1.4497075505741523, -1.1660183149977592, -1.328091134873811, -1.103545623544987],
[0.3338961076774624, 0.015683907518298446, 0.3927270357122398, 0.2757392283147375, -0.03153315292439125, 0.07171740705054183, 0.1684972985901707, 0.31844287558855777, 0.003957607452559693, 0.40987447762801354, 0.20773941505834154, 0.18352829680674346, 0.10884435742938266, 0.09460112202612628, 0.1149624243838254, 0.1401046215276004, 0.40059131603190945, 0.502418047984553, 0.24731700417226074, 0.046794582498959084, 0.060991554830254, -0.021412514079210576, -0.4279098370448915, 0.8620283852573036, 0.5031012746634684, 0.21666331871740702, 0.07538054654278582, 0.05156811447104626, 0.055555852463016364, 0.87162630525387, 0.6241010528568174, 0.17604424002723784, 0.5505869232445958, 0.36360700973420934, 0.2605878006412978, -0.6564373798116557, 0.09115867693946088, 0.7476049594278735, -0.3449705008969887, 0.10431203253986472, 0.022472891601163694, -0.3672646237753108],
[-0.022099686312801826, -0.1106700732205772, -0.020486212510999406, -0.14539883577925217, -0.12988072618459712, 0.09776312506245585, 0.07343438992072822, 0.09832731230509024, -0.10249924307235514, -0.49798453075393406, -0.35807118823244344, -0.04203920834842646, 0.05002854495605662, -0.12795618909549186, -0.19522336431046852, -0.2798998848288397, -0.20116348149452448, -0.7409412963576272, -0.2440650355344621, -0.11406276600557154, -0.24242304135356926, -0.26526236771860096, -0.2882093780978116, -0.41684657639457584, -0.6678410410226776, -0.22748309380040355, -0.29716346525555126, -0.1850255238519862, -0.5389799550029303, -0.6520351313936074, -0.6336922990349335, -0.5746384373120178, -0.40963771232537016, -0.6860951749214492, -0.698169956213061, -2.749039198374469, -2.2904813642061286, -2.6903207863913807, -2.7537404784416046, -2.7476459558350097, -2.3143778955826004, -2.714949591481753],
[0.07229020129004811, -0.019063616013647366, -0.13926115241088616, 0.2030584771278814, -0.007864259479063144, -0.17578604039891224, -0.05962736214627579, -0.10948067669246449, -0.050611877686642284, -0.23871142522591451, -0.03663796673054215, 0.09358105354578634, -0.0978106369005348, -0.05601008187250382, 0.0564265211100654, -0.13502986591116062, -0.5797105626690221, -0.06261718480952935, -0.15629024721534726, -0.17484233570513738, -0.33846028553938784, -0.28866609752209843, -0.3537479306880096, -0.24315974410516733, -0.19004032804911972, -0.4213522010696292, -0.5241398510010383, -0.38860739938122796, -0.2834534226903044, -0.1799403504107147, -0.49247656984428656, -0.43099952579995376, -0.23174800877688892, -0.25343958629442453, -0.28848911456458237, -0.6773453564600362, -0.5586251203319798, -0.16893343273013445, -0.878785311682143, -0.5979851155955839, -0.9197361594913536, -0.6261239837511225],
[0.010589851776949464, 0.052665777518469646, 0.05305608004222978, 0.04180574535329491, -0.02209935369873509, 0.0733416891620386, -0.13490899701276532, 0.07853621721792188, -0.06670474112940648, -0.21206991455691368, -0.06602431717526165, 0.09027996403392992, -0.23232914188604661, -0.11554635263704183, -0.05726769533418646, -0.13919905505199343, -0.25013096937091694, -0.017667364373805016, -0.003440691261866033, 0.06271750378846153, -0.018874672073266235, -0.21727638187970344, -0.13442843561228637, -0.20168703409352895, -0.6271013835591894, -0.16896188589047723, -0.21657937364184587, -0.24257568332156992, -0.5845386171089876, 0.330735285609604, -0.33514517774686114, -0.6814066382390233, -0.3251030061040067, 0.17425461570358738, -0.5269962945823438, -0.36746002199851835, -0.08010829349697343, -0.9671004671256863, -0.5799375704638743, -0.8853019816498175, -1.046106357204338, -1.1923969444127231],
[-0.15428125472532336, 0.26610028501469984, 0.08164579425974328, 0.028361835024261435, -0.13858372631917376, 0.016453834632413494, 0.20324700568021792, -0.046684231906217906, -0.15213172266546904, -0.06168093462391998, 0.07259382645885541, -0.026169705389205652, -0.30725236488105584, 0.15776678220671905, -0.2436791699323647, -0.049420951004949115, -0.13372776417282897, 0.15454111227358946, -0.3203676669894523, -0.2830625632882081, -0.1187237059402841, -0.4066753454429399, -0.23604028194811888, -0.14490935656588558, -0.13630087768911273, -0.17755670485267233, -0.5410676137091166, -0.2259002700079449, -0.7892785879367817, -0.4424044904731202, -0.33678226478495077, -0.4396268737810151, -0.5497926985432121, -0.5165897911728792, -0.756063716253766, -2.774104960121779, -2.5110909413157954, -2.946502024583473, -2.5986859644815654, -3.0689107723537075, -2.804461442503349, -3.062105262378982],
[2.4008708502462546, 3.2473896464297787, -2.4005266599725696, -1.8232985229697727, 5.6043639167451405, 3.6869768922421415, -0.9674369744536985, -0.21345510243014104, 2.231909841062153, -2.539135560282589, -4.004773127602762, 2.5485160106207214, 1.5703527244872928, -1.1004997351517556, 2.6228492057126274, 0.1060092480943702, -1.4185796318812511, -1.217610953201152, 2.51544799390101, 1.5021902491889139, -0.590584215731608, 1.3359983321761877, 2.9907470847395525, -0.03910940557396953, -0.573097362589214, 1.8503735521024485, 0.039699941978870654, 0.2199065269878352, 1.8595596297992558, -0.5733945779392583, -0.06461382356767828, -2.489335198941177, 1.397052363345165, -0.6732820383262006, -0.5634945139820029, -1.8694850357213448, -2.146202501423711, -2.678459421950294, -0.843115867104896, -2.3341248959079355, -1.7952305838609366, -2.6001027629627758],
[18.778855313914832, 1.2293381181513925, -0.21112256088817857, 0.43158928845145483, -0.1918906984415665, 0.17574085061901218, -0.5876735050402191, 3.2046794567415327, 0.3171885919119403, 0.07507181737579711, 1.4872446936317043, 0.36806402173199537, 0.4937794019513054, -0.5724151719681537, 0.24041188209656517, 0.1955776244975368, 0.13124158869660674, -0.6074414527076676, 1.1999509442748215, -0.2028947164608228, 0.2347467864635334, -0.16538038937553592, -0.18757669850747954, -0.6982788908633754, -1.0979121499498519, -1.6960678157417246, 0.07598660764921415, -0.44184193743975064, -0.43475313121924436, -0.8922343690716051, 1.553086249537422, -1.4204574879130154, -0.3838801961719422, -1.124594128473371, 0.6289399872442097, -2.566563378843508, -1.9401172461320955, -2.2968863754937248, -2.1996214624502595, 0.5251521293378346, -1.2510413591891654, -0.8227630640460339],
[0.15655396137359665, 0.04875523260045275, -0.019057563854824527, 0.08122053455034162, 0.34005173584649717, -0.042594420397413194, -0.35384211664560644, 0.18373891722155378, 0.003585786156615132, -0.3260793179692121, 0.3441255167664634, -0.23269311737960988, -0.012078075612904674, -0.42406484559843205, 0.0068965815878483236, -0.17107088633152248, -0.3653767184639178, -0.07217840647637927, -0.5208845023428392, -1.0714527864237364, 0.10084739068391707, -0.29151141091347654, -0.6396520369327314, -0.20718822023904046, -0.5811612999267883, -0.41745581319206415, 1.3674193797605527, -0.04533148344580396, -0.4269599598962735, -0.735304417547707, -0.4565057191738842, -1.3549797466498221, -0.4326077071196643, 2.061456666006027, -0.16821283972804751, -0.7008285250796453, -1.5322129150055495, -1.1609925495434974, -2.0118306178037426, -1.1559452892663626, 0.25976521196703234, -0.7981454265732565],
[0.14519481010675425, 0.013082428860601767, -0.23324442811548002, -0.17428669923871895, -0.164771397501916, 0.13655201663277522, -0.013363543964498095, 0.1385239885123698, -0.17160998406783598, -0.18367468413263838, -0.10927829288419065, -0.10764843785181219, -0.08111012343985403, 0.04916650150593924, -0.46806484830439676, -0.3253689692467676, -0.26280931535193697, -0.1790150037469047, -0.21992191678497572, -0.13056364932101147, -0.15636928258255658, 0.04212663961557478, -0.48719824946081014, -0.3369207530587706, -0.15306072712611535, -0.5114697438562034, -0.5315884448833775, -0.11930132565376578, -0.44351431962868904, -0.5004434267011654, -0.1983150868133201, -0.26337079136211816, -0.3819736480005999, -0.6982040491556679, -0.3725389676917827, -1.1468802185061644, -0.9513291799339033, -0.9860770175157577, -1.229204206776856, -1.2103029510996592, -0.9066970888183332, -1.3576530042617132],
[-0.0019752233740109875, 0.438238457040114, 0.008192212446781227, 0.4535252401511255, 0.2718388521939639, 0.22898771263552653, -0.1438633067914361, 0.08735580029797797, 0.1970686806279674, -0.10391043074768729, 0.06174871353784658, 0.006378367213981241, 0.02041649661607518, -0.19352218558935397, -0.19606457525321425, -0.05173510781565291, -0.32815982929326776, 0.2713088646850812, -0.05773659675229632, -0.08549582206291499, -0.09932219733630376, -0.1666152250422577, -0.2583813276493741, -0.06398416463287493, -0.021116488877112986, -0.264370971232243, -0.2030246148259449, -0.26821177727618306, -0.7570694966383953, -0.6475581862666083, -0.32626118492320483, -0.3760981503850984, -0.44186788260229526, -0.44300333849211027, -0.852315612654029, -3.4260427034415355, -3.0639128556446136, -3.517612971294833, -3.3960123319664266, -3.597066358018193, -3.333073562766969, -3.461318677161222],
[-0.20431228341666086, 0.07407421805064215, 0.05909015584820395, -0.10427411549822924, 0.16378449859470723, -0.03767240061750668, 0.11396983138010237, -0.30617364554535287, -0.06220658632047882, 0.1459707790864923, -0.3726214676072285, -0.12917197935311803, -0.0470367409931102, 0.16201785837668656, 0.10498886168174933, -0.17191547992309747, -0.05871518625667721, -0.36693929845843304, -0.34603858861077685, -0.16942595371827743, -0.2436445605885482, -0.042876792835427203, -0.0062468651250177165, -0.2352499988432272, -0.21070668597310946, -0.34969995432505446, -0.454115292166024, -0.45225189970034957, -0.47377841455407926, -0.4143278749355651, -0.30496661776312994, -0.2484686289060987, -0.4213287074082847, -0.7576674087682236, -0.39133054806982187, -1.099305757379632, -1.1577184076716691, -1.5211418238247336, -1.8157678622899576, -1.3772425454850377, -1.3530344592854882, -1.583597441073279],
[-0.06712763272057054, 0.12201975158496414, -0.3639248051148623, -0.11468851312504343, -0.2261517289266024, -0.16777523152101267, -0.004569017759386876, -0.04675436321869637, -0.09330884769083747, -0.28414235390076203, -0.2619232013956704, -0.29294454702130224, 0.027832954305846248, -0.09236454361430604, 0.21165478768315543, -0.1923409192190433, -0.43089997049625267, -0.35421593072022123, -0.3931201274775086, -0.32743833302720676, -0.49208921344660667, -0.1357445173009963, -0.31868691441037983, -0.682203610966974, -0.5421904690938579, -0.18273521959580552, -0.6056298854194524, -0.06147412717652624, -0.07356831658797866, -1.0918141008803441, -0.570671653041695, -0.14623347377633553, 0.22428391744311177, -0.9922322675715142, -0.49338412910233903, -1.8411860561446618, -0.4386244634597779, -1.4548176024796584, -1.4665989441760083, -0.7672425689600099, -1.1592763394895536, -1.066532574899265],
[0.26914893490879405, -0.2380225684158656, -0.6853344970264823, 0.1353147930639464, -0.05868862129781078, -0.45816572149471935, 0.20893281557274457, 0.19086150824556386, -0.5313252159708175, -0.8143285788924418, -2.1038976217488057, 1.9181291028797007, -0.7038362398397975, 0.6552450874899428, 0.33418330884454467, -0.8008318724933874, 1.245940641461886, 0.21015654732514757, 2.104604933457101, -0.0492956843313311, -0.5298850289436611, 0.2523902990199572, -1.0115103339650646, -1.378978801996893, -2.115462327004135, 1.5744211907877124, -4.993199988696358, -0.21781415330988316, -0.7151821968273626, 0.40274767809194867, -0.17004484187434216, -3.278142807337116, 1.2676029905987658, 0.5166067657365506, 0.5474812005625016, -1.7696703912373595, -2.823667878550504, -3.1096627777104944, -1.9468801458089555, -1.0909584880095469, 2.424987221463762, -1.0618181902326533],
[-0.19359969467917681, -0.062443254811919084, -0.37592987828653623, -0.5381003018634355, 0.007827025536234914, -0.12454861785746627, -0.1473741169247868, -0.26961346805507513, -1.2985507196708608, -0.06283760109880765, 0.02757617504521405, 0.3149582920799668, -0.5123389308586496, -0.09309838368472612, -0.31980531312308697, -0.029908464741029706, -0.24759677370306116, 0.529740488140087, 0.506468498558144, -0.437072065445725, -0.24408923252720374, 0.06457933463204237, -0.3942425647869326, -0.3493457249707809, -0.8041682728315628, -0.3910616797919754, -0.8883294640410175, -0.6868497016916066, -2.352066406792309, -0.2525985718124081, -0.5474930366294871, 0.54887946974511, -0.9430271971000057, -0.3874238717337011, -0.9575466927436195, 0.059900749549968255, 1.8306505454759232, -2.2706251504498, -0.5599008145876703, -2.136950376496283, -0.8525486693320106, -1.3386869499364502],
[0.375856758957369, -0.04732603722631368, 0.021898750082570306, -0.20636908165443824, 0.057725227820960026, -0.25287984108096756, 0.02371116450363314, 0.10212559216907632, -0.060609579509006604, 0.11964773967411368, -0.4476062629296751, -0.1871296244729145, -0.20531257354571797, 0.19106163733936055, -0.05222994466625285, -0.27114009413600837, 0.1771153621527492, -0.5984840179113478, -0.0044140765468828405, -0.4235501066649134, -0.10940349737605924, -0.6314415681592579, -0.6101404182689933, -0.22138638645856187, -0.7811077071590083, -0.15202668692154697, -0.5961463718004554, -0.04357425660397056, -0.8823586571089581, -0.25502788151342265, -0.4453541259773948, -0.35054925642933826, -0.464164489331461, -0.3939982111903005, -0.4442081058718106, -2.7012599651987954, -1.9870403750936008, -2.164239208519863, -2.6630061000397207, -2.022706767794447, -1.8421250863093854, -2.3455061475399064],
[-0.030710530042611487, -0.12487898534967726, 0.25047100612758544, -0.1389135880019234, 0.10764971457359998, 0.07221173247760465, -0.1831956951540276, -0.2804128710813501, -0.2562525608298664, -0.16137286461690958, -0.13218941892756786, 0.18876839351060165, 0.21618841435056768, -0.2033249512102816, -0.25002565547827305, 0.05841308247988283, 0.05453357705426616, -0.38073292442014006, -0.04653302853009755, -0.281514234073661, -0.10681733829563402, -0.6020088298633987, -0.12033845195985013, -0.16883736149745826, -0.49166896772776203, -0.3824857036934411, 0.0021600182400555975, -0.3565887721458492, -0.7521359942971869, 0.03461154655813451, -0.6509255217385611, -0.35213439530597296, -0.6006853023364767, -0.2917038419640657, -0.4320552872668991, -1.526963110127062, -1.3713151549210285, -1.5498172168890967, -2.0269166425094425, -1.641019637575899, -1.2246537239293867, -1.5569024260549773],
[0.7284146769109607, 0.20944678129946856, 0.42752820581233447, 0.5649024304404106, -0.36744489802068125, -0.420747762940607, -0.6147641352263763, 0.17437089459932284, 0.047738134740093544, -0.1992120496799312, 0.3347817319253397, 0.8209793393247509, -2.5703516166977534, -0.7099580361585452, 0.19239601884443475, -0.44614334381820614, -0.3071899137165655, 2.40707937910018, 0.9301754217108298, 2.415558469299861, -0.43328752561030753, -1.0877360044793551, -1.3253298870179004, -1.0995537550340748, -0.8795586250169303, -0.8020835371620861, -1.1869714824671038, -1.570766217188806, -0.6475324801268568, -0.5624604758007906, -1.1545445311814817, -0.8613408840290815, -1.4319349943878192, -0.7218883736853642, -2.0527108683697057, -0.8081263699251063, -3.245643020489998, -3.735016370929331, 0.9517836907014534, -2.528918073304669, 0.09294976024876789, 0.8766850923488172],
[-0.12280799697050428, -0.1268480381098143, -0.05577265708886903, -0.1994976604003446, 0.03984240587078844, -0.0011062351032140366, -0.17202187124952875, -0.27599588760400545, 0.1346516387604295, 0.17372964788371095, -0.011427670538371688, -0.1664690988088344, -0.07042174769691212, -0.0973979424710176, -0.1356619002294065, -0.08785755053521453, -0.04113369598377779, -0.317862009520869, 0.07875548627372984, -0.22577511928265961, -0.13146343782797026, -0.16763797647535886, -0.09829768552608735, -0.18036724021768422, -0.3240213497447809, -0.29229225861996405, -0.3100392045618243, -0.3869351932298427, -0.7076081338503489, -0.3326035758321765, -0.4305843527291608, -0.3754876635055537, -0.4761927225975288, -0.32974183975209576, -0.6617382986417345, -1.718707869175509, -1.596735526370133, -1.8537817467002198, -2.4008809282524934, -1.7720659683628663, -1.641658545623078, -1.806652816276762],
[-0.13773517503052204, 0.0867111136610605, -0.24096043827977567, 0.5434006653560718, -0.2003509487271135, -0.0073980651856713, -0.08732049633862275, 0.2300104782782617, -0.12161679679184502, -0.3739524379497104, 0.09333895681581382, -0.2172254994289256, 0.00197215744290826, 0.02145572366518297, -0.16040574404142885, -0.2610933540968891, -0.3041978499146538, 0.08819629812853445, -0.15809995677001495, -0.041430772881542906, -0.2560100739004482, -0.14146844931006913, -0.5485867563479447, -0.11819195441977065, -0.02049544907700174, -0.5429191751228463, -0.6601409376974703, -0.39272532141310657, -0.7217765403651704, -0.2630158860935292, -0.5109400051934646, -0.48242619819709087, -0.49262690301854684, -0.3429054005604545, -0.8311349915822779, -3.273887344644206, -2.975603705112051, -3.3010612988466477, -2.819824169287097, -3.3928252851205487, -3.1597066289588636, -3.3994012606177932],
[-0.31917122709013546, -2.1567448732631744, 14.260462883170526, -0.06520191660474293, 4.206819118158539, -1.4242310489883063, -3.677498018982608, 1.037063267692351, 0.8324045937317889, 5.841933110121954, 0.07562591237384082, 2.61377555454925, 1.5372374645445388, -2.760383229017477, -1.5527217046651542, -0.21944492866958268, 0.23829748239926477, 0.3647084113681593, 2.6973145816509754, -0.38250714365576166, 0.9410184335017403, 3.434272584145695, 0.7916356148043457, -1.5917456119748794, -0.4671283938783893, -0.27145040562112527, 2.530918927856247, 0.6671656287951717, 0.7703452171750284, 2.802526254809262, -3.656000590721914, 2.2614982593976682, -0.8913307730263491, -0.2453675521426591, 1.5569791033011116, 0.11328522814853301, -2.5447791990775266, -7.906769331626167, -2.5223309201867155, -3.3178547652976422, -1.2850619266008685, 0.8705496861726896],
[-0.09991974585517818, 0.04909165992051127, -0.12722310273496218, 0.11758120781151658, -0.09525199912599258, 0.13200164690559574, -0.09000917314679262, -0.20284986685260148, -0.029492877880863293, -0.2569076827870056, 0.4971243789430792, -0.27038313001605135, -0.0554294357027649, -0.22420231798972526, -0.2703495458819898, -0.2701290737407719, -0.031846811795995404, 0.20427223604537176, -0.39951654226853633, 0.027556115212233865, -0.32537137258637056, -0.18561880072804968, -0.5612939220848424, -0.15467172112746505, -0.11181891210738697, -0.5314361297071629, -0.10386189460716325, -0.20639366840597798, -0.30254198586079367, -0.5310709477975248, -0.317938873673905, -0.6356423891390941, -0.548177259936285, -0.21143757403600516, -0.4030615833873779, -0.43665278242828676, -0.9561300287921889, -1.1876297393603896, -1.0789591500372535, -1.4565609695774207, -1.1712624606779092, -0.6588959965423615],
[0.040589370852283806, 0.09301334571303667, 0.2122837398208061, -0.007668335367228786, 0.19410580491498924, -0.1113043320698292, -0.14380991464389267, 0.040577667310081854, 0.23452970279032523, 0.22868783365444212, 0.1699486996410198, 0.0942015272068196, 0.14493429802431512, -0.11127444079157724, 0.2224351060595668, 0.2687400537379781, 0.18661698049634493, 0.1332555210705999, 0.2224009374999702, -0.034324635030319114, 0.2739084816901352, 0.341375098977316, 0.4828744366537667, 0.32360901315017304, 0.03408807308339993, 0.3277446708439729, 0.321066249722002, -0.081793282574332, -0.007251664520285727, 0.6633288258284985, 0.1724604126004005, 0.2619457529677235, -0.02117352689197513, 0.2736761774440387, 0.052566060215952774, 0.35482626543674034, 0.5400985910262031, 0.008511498859665909, -0.7219567790002148, -0.05247852858943988, 0.18692473214218944, 0.02516890670571677],
[-0.14355335165926053, -0.023659770936052604, -0.36829841708400474, -0.6320966145769928, -0.5074148017905462, -0.04017126311443724, 1.0401123662846463, -0.07056562568739548, 1.99379928953911, -1.6509279857362624, -0.6407724234489296, -0.74609842198621, -1.3945090959518616, 0.23073755813415522, -0.8529892260324803, -0.19350690472933862, -2.1169348727381596, 3.275621905228111, -2.3302262659837965, 1.093656364712512, -0.004813851554345592, -0.399140364838129, -0.4964683566788402, -2.777004774145328, 0.42210529547563536, -0.6666011170500462, 0.32319886214178145, 0.08996604452125244, -1.6412893042431829, -2.468676351678444, 0.5336591324590593, -0.8207361884168944, -0.43521289896701176, -0.9654935338002749, 0.06113579885701396, -2.6850349661191975, 0.5256865196534822, -1.1830453103224976, -1.5976172438626746, -3.208052997096462, 2.178039025507473, -1.3946390933844273],
[-0.3075049489776265, -0.17473582466603296, -0.2568140387458126, -0.5833998671315355, -0.32839132069675847, 0.31277409953830154, 0.07512993149334783, -0.5968614860491731, -0.4968031157904213, -0.993110810766564, -1.7857726642169411, -0.7346659160939494, 0.6904930793061901, -0.2142770844828415, -0.806613226405677, -0.49177187166236686, 0.4315592527944061, -0.8449394572635561, -0.6864617443366905, 0.6751148971064406, -0.4208996170213983, -0.7554514986388281, -0.3195953971555181, 1.531066183998623, 0.9240673926438798, -0.10667001121231348, 0.9237208495138621, -0.29890814898127943, -1.1424111374208028, -1.4802132287600167, 2.3345594166872665, 0.6807055606340479, -0.34323370640795353, -1.0844480827127991, -0.7500275618551868, -0.9908054485522254, -0.7454077571474409, -2.1844771154569793, -2.233197430606112, -2.586369260181464, -1.6783388174511455, -2.068821817517292],
[-0.2947850363287624, 0.39962580784318963, -0.22956663921183362, 0.014482064813766996, -0.16305890058168943, -0.07063593140818777, 0.13587192475065613, -0.3578585326795244, 0.2953295174594043, 0.6867836493015461, 0.08408171645416193, -0.37450302220280673, -0.20028641240000786, 0.0734005635625387, -0.25910448624004756, -0.3692308943659196, -0.9488806255123241, -0.3389567960014983, -1.1198185606887123, -0.3333796289804715, -0.45007225270510187, -0.4867275775570464, -0.6106782265463641, 0.6166882338462896, -0.25787003465328395, -1.3412726993375137, -0.7471416052165568, -0.0006816455965613619, -0.4685611317012443, -1.2006073271175373, -0.442428893187974, 0.9884692983217894, 0.31040957939685265, -0.5192895077355237, -0.7567321254331713, -1.3623002820563637, -0.30447183883744966, -0.006983914069278628, -1.1740598027889957, 0.36551803275735795, -0.8155436838044335, -0.7975308971386457],
[0.10689734134673003, 0.0010846528593984447, -0.38444854514974985, 0.02407086073885475, -0.04911739732753759, 0.21003359623048953, -0.11575443358139832, 0.18601196689720045, -0.08558494470909263, -0.48332003894847686, 0.035820756526142264, -0.06565374089105477, -0.07789352181822397, -0.30575797913917263, -0.5599719710853146, -0.30443577827873025, -0.39643242896812964, -0.27404490632018086, -0.007616003217092015, -0.1348422447623586, -0.1190247324269148, 0.2548207994388252, -0.7474996408766637, 0.4987801884612451, -0.482039335347705, -0.05549551273992591, -0.2364149083581731, -0.34391130002605813, 0.8661280493080595, -0.47223952404864805, -0.20200314600707475, -0.607772451387459, -0.5030648784131251, -0.16004975867832347, -0.5106909401499131, -1.1947268985805, -1.2490627685057694, -0.2645654809026479, -0.9509846583783818, -0.3507601967059201, 0.7118093680221467, -1.0172117020479785],
[0.16660748303206963, -0.313600905397008, 0.20698301378491277, 0.3332643051254805, -0.17965785715557717, -0.24166784357887477, -0.15773303584214562, 0.170757295534848, -0.07396264678649428, 0.29220027670972626, -0.022989487732959807, -0.23595052843688108, -0.21860756153222727, -0.052594390756613325, -0.18355504986011864, -0.4540067446038844, 0.0417728615194723, 0.30570238108584663, -0.4280251158491488, -0.5040114656705997, 0.10496404845947088, -0.32379205405316713, -0.729794568746686, -0.08427010458284881, -0.5247422756574419, -0.6248203995875034, -0.7821602430025629, -0.2770169552360844, -0.8725699323680534, -0.41619941270931377, -0.45251986029654967, -0.11829079029838542, -0.6830432562397699, -0.24273507642450626, -0.5205326845320185, -2.1985183217220747, -1.306266814020042, -1.1810156319754952, -1.9227737394872695, -1.1642377053379376, -1.0654528295303727, -1.7311650628463922],
[-0.08187784254450328, -0.147973652625407, 0.08064918703418522, -0.10828986631490796, -0.0782184512277485, -0.008615078185194736, -0.0049596682087776735, -0.25211223543654926, -0.03174637757858446, -0.05482947379043317, -0.1787137570307674, 0.0006811593359792908, 0.0522982634899303, -0.054400782362838906, -0.0535607176299553, -0.07697376264337623, -0.08750501032327404, -0.33020945059814805, -0.024721688784275175, -0.061518745609356745, -0.20792762108754326, -0.2746526482653775, -0.29210468138889556, -0.06557830589395182, -0.32903754905766736, -0.25155365677013636, -0.20811752258088193, -0.305579096148234, -0.5172710484090253, -0.15297309055827754, -0.5262577832078513, -0.12990877182371727, -0.3353525964447443, -0.41572448705735, -0.5412032037172545, -0.8297804019404295, -0.7989783837658565, -0.422898174840972, -1.9906768331739106, -1.068583451728938, -0.7603530645661853, -1.0484538017249814],
[-0.41598116904259164, 0.14091839414574414, 0.032365784135043825, 0.04449006604913536, 0.0911625611842973, 0.049775146052195955, 0.12855471832605816, -0.11148990000423308, -0.007069444092723661, 0.11250006788499248, -0.0010176531383286917, -0.1427238776159451, -0.19251936097192224, -0.047088644825658464, -0.4187929697653803, 0.05562908809150056, -0.046533005726940656, -0.2084864637727638, -0.10496447038556211, -0.021254710796574292, -0.050567421408981524, -0.2328733427100392, -0.09903799713910233, -0.1697370449249316, -0.18670543127436107, -0.35555006471782025, -0.24681462562596346, -0.20794225606595013, -0.7493074422811267, -0.3498083743411381, -0.4797421172976569, -0.5600491818586273, -0.2732752577028177, -0.5892289832138388, -0.6935109837520826, -1.5047957370969915, -1.6747141092495854, -2.117583607337566, -2.3960142901152177, -2.1029212103781183, -1.955885671048535, -2.1727320519131283],
[-0.03160065325854583, -0.34053973063137954, 0.07069394602986877, -0.0671251774165756, 0.7861542090299884, 0.07524730318931575, 0.8292252019599538, -0.622579418357566, -0.1818600346099754, -0.36533817805478436, -0.8121630610851163, -0.17394412408313611, -0.5984988085609126, 0.531560458717102, -0.27851543090964875, -0.44003818686879054, -0.15316307840471116, -1.2928614938154714, 0.2927513392704186, -0.294686130671014, -0.30894210864166216, -0.5906801231157247, -0.8371078531506979, 0.04187882080732047, -0.5096754037526224, -0.5750733954318215, -0.5589329896128766, -0.7820795838147576, -0.7380699836367274, -0.43028914447193956, 0.13358014129057713, -0.4150475174595225, -0.32721618400773683, -0.2744442639017602, -1.0073973972217625, -4.042865818966194, -3.6084420994590753, -4.010756206816895, -3.8713888042797775, -4.063015839743674, -3.77080568987372, -3.976686071025393],
[0.09928460807575733, 0.058784301944254526, -0.008708506615754813, -0.15730373679306564, -0.21976768654868525, 0.016236351014451998, -0.1946604591623275, -0.05329004509870587, -0.24641221508982591, 0.03383126421783013, -0.32819875228543094, -0.11797804186531978, 0.11317907814108516, -0.14877926111727402, -0.09315415723414447, -0.26702998390173094, -0.08210379849383398, -0.29441057501097834, -0.15765786963859207, -0.26333699644877245, -0.2272691046796766, -0.21058506730183654, -0.2898409615386109, -0.2508781757686776, -0.48509076835153225, -0.25038890925613594, -0.12648433166278344, -0.28649900155786445, -0.7197818860801134, -0.564018581104854, -0.4988449458480281, -0.061134793751448285, -0.46852701213162284, -0.6981576919112612, -0.6827331417325603, -2.108799550229147, -1.7908510439533174, -1.8458382977274816, -2.708006543544391, -2.0423389603886983, -1.7997939082723153, -1.9982020023498284],
[0.4695311203065954, 0.13341286308784384, -0.2533902846065823, -0.044641978346361175, 0.1698466480544109, 0.14766639578652802, -0.19730458417410246, 0.3297267191357025, -0.016494410108966212, -0.6241267585758478, -0.40037309711637215, -0.13144466007306166, -0.47620754013094263, -0.26650554826133716, 0.10891145606234638, -0.014422752319262986, -0.9183027398904624, -0.5476496860403512, -0.014439155009513545, -0.42350854667060034, -0.40971596796244136, -0.510944342041182, -0.2244407562148779, -0.7822975841771385, -0.269225495181653, -0.5693786921123916, -1.1284083530940177, 6.107561072746631e-05, -0.6150693441439777, -0.6364773858658274, -0.51590350137053, -0.3767869539809972, -0.6542282773708809, -0.6006680382618171, -1.0323702160988788, -2.7962999786958433, -2.0922811068834957, -2.568897243280808, -2.3361554594002394, -2.476686370408975, -2.25753592968188, -2.166150537762718],
[-0.14716764967639315, -0.4274077913474552, 0.02388704866975392, 0.6733623182187268, 0.8417824060924921, 1.2918275782350133, -0.5946340197938366, -0.31787215205603825, -0.7984826834869267, -0.14771402006200082, 0.0020414169440062155, 0.18929513014946078, 0.6550692137329372, -0.33678316829617544, -0.2532615586342401, -0.9579568128524475, -0.5190007808280261, -0.07137897383023714, -0.28969096513853243, 0.6596437549137492, -0.005132756061077863, -0.41124844233597335, -0.3793136980274046, 0.1554227597136202, -0.5907322459881401, -0.7950410638553044, 0.19702317108349301, 0.264923414655786, -0.5260045477928762, -0.7219771996856991, -0.5796340306309585, -0.15710001308424387, -0.7796758254633901, -0.28019059054402823, -0.6563898889912501, -2.4266205882153984, -1.8898380319619401, -2.309195016769506, -2.9293832320167166, -2.8371365547176817, -2.1815043742379547, -2.2091061442860886],
[0.0756625551306643, 0.1545853899695135, 0.007773795909812031, 0.030177545621363037, -0.18543066215132353, -0.03391032993778687, -0.3699201929830884, -0.1526761271114478, -0.152883042623332, -0.22635579953430973, -0.05364511194594272, -0.17633987947071617, 0.14826257144964847, -0.48499204967221915, -0.23822449512222996, -0.3985405733000089, -0.06877452930443294, -0.2235623686163213, 0.28328815524382556, 0.07360815023348684, 0.17972293637468395, -0.3172606576210015, -0.4617785082201558, -0.6363157959552854, -0.3522511150469382, -0.24612450949809625, -0.2999027250747742, -0.15668933894276704, -0.502103686372694, -0.7419396282941395, -0.3979126438825285, -0.7727158416343894, -0.2626608801973785, -0.44450508700588903, -1.203296528188245, -1.8864183557790368, -1.255545504858382, -0.579757307436758, -1.0140303481647257, -0.18040394157879203, -1.0551587016106334, 0.30887231044173624],
[0.13460364482796583, -0.35294302001590766, 0.2702674515819427, 0.023608246248552647, -0.020132334076578783, 0.16067234387811222, -0.21811446127189807, 0.24156869057052852, -0.38206771973485754, 0.3180103904309748, 0.010060052325236596, 0.13533785931707545, 0.027734476639695044, -0.22660872443949345, -0.07960668829221566, -0.15577638287916398, 0.04322476562437784, -0.19160146453892618, 0.018967521010983167, -0.1926016722665214, -0.16291435445286154, -0.20093727647477547, -0.6666568161529103, 0.05338512340515619, -0.18798833974024703, -0.4138658534507829, 0.14306362654471785, -0.44462472294502686, -0.5823990292014215, 0.04125838578180563, -0.2566448968505922, -0.5323526447160084, -0.5234890300020476, 0.2548600752199742, -0.5276382243514514, -1.1424268963725137, -0.9627515896597929, -0.7355347241612047, -1.713485145367849, -0.7116457531126309, -0.760589787539177, -0.8943499703812071],
[0.04920374225073611, -0.03966764245136223, 0.11712303205990117, -0.045198558827139426, -0.33019531059516377, 0.38908649934208217, 0.1912749653366706, -0.0973547901417122, -0.37085750753168784, -0.041425891606158415, -0.13791806596140976, -0.3605932465788778, 0.14478657622817526, -0.13693867709896387, -0.3835415659211785, -0.394590924940751, -0.24956913084773819, -0.23826850153245602, -0.5074916185143373, -0.22238322570370367, -0.019293845097746478, 0.1224231962645514, -0.7723007872595292, -0.46373927934445897, -0.34483068162962, -0.7187988006668344, -0.0022208799592217388, 0.02135278848365321, -0.680957071431593, -0.791317426697693, -0.6147963632789427, -0.3702848708076324, -0.32805532208129706, -0.37239114784598426, -0.887505552529463, -2.2342090434887103, -2.0347748515702784, -1.8915202143914756, -2.4153406961554853, -2.012080614540643, -1.6676944633363169, -1.5626433677883647],
[-0.07835926098968628, -0.09891107439949319, 0.08534437961194155, 0.06609689469453696, -0.017464527103522667, 0.06280951013349294, 0.147249161392401, -0.07118478635012403, -0.25788330029905865, -0.0536671756472483, -0.22615525688797597, -0.17582370883679238, 0.016654460040451652, 0.1950696444853861, 0.03345138877634691, 0.08221769252797008, -0.220344853782776, -0.1661151594073424, -0.1821851749641519, -0.06278710783624461, -0.08612874069286011, -0.19792118831566233, 0.28685579590418425, 0.04612703799746968, -0.3420452225686175, -0.3229909613409441, -0.5244729577888044, -0.17556626756114987, -0.6330752152435445, 0.8230858279276992, -0.2794357714560409, -0.4304692923737556, -0.08630710586655506, -0.17359484810616763, -0.25729898451377, -0.21436944927378382, -0.017971253881357882, -0.7376153523285172, -1.0549102827768793, -0.7482072938886672, -0.9754469266384188, -1.0080279054450825],
[-0.14307790851600508, -0.1244073028078813, -0.21544331990072732, 0.052414282461365314, 0.31717184656691383, 0.25308192859754974, -0.37661360722525256, -0.06898494731850614, -0.4471530101448833, -0.6139914938542196, -0.09970724886901827, 0.12346168858166487, 0.21364578421691438, -0.6789553466566288, -0.20309073344364773, -0.36303149691079095, -0.285103503029632, 0.22637417841279506, 0.002327293796252999, 0.3665316741573684, -0.5967747495563008, -0.3973569979593585, -0.30009965533219995, -0.6221769011321543, -0.07471062280856502, -0.24473606800067438, -0.09767762439095257, -0.6795489422342171, -0.2655264776823217, -0.7337114912014554, -0.4559862764894729, -0.07053701950947919, -0.389218560875769, -0.054096819077285224, -0.606827329251635, -1.0731631278201301, -1.221354228977336, -1.09094552190337, -1.1825213791203066, -1.0263931684756769, -1.1935837109451826, -0.4843397900709336],
[-0.19384619035295111, 0.3723029598571085, 0.2190571508030989, 0.24424728696217196, 0.2871331268355351, 0.26140701830780055, -0.4378536354796128, -0.19778370222741215, -0.06594849536272859, 0.11415912155819734, 0.46201035977228927, 0.13916861017264961, -0.0004298305632392943, -0.4609889876792058, -0.24581971797832594, -0.0935168783776724, 0.05624940003800805, 0.125108758332194, -0.08843627720923755, 0.04778053327761917, 0.095205697938697, -0.05222013278140258, -0.26126377210341845, -0.13191556924791745, -0.24215032804996536, -0.43752385257038934, 0.007161381355327881, -0.003544376584547091, -0.7910275665430057, -0.2797329742913649, -0.6609953353819746, -0.36989800100925346, -0.34211959998722, -0.18595935174129952, -0.5065811875985538, -0.9228112473873489, -0.9360872834240102, -1.3363403477036981, -2.00690942908402, -0.4228392316110277, -1.1221043476890868, -0.046866337762648244],
[-0.14137755823291145, 0.5387118701194512, 0.35447532694456135, -0.3561773414133026, 0.3273105810985308, -0.3455547298378076, -0.31866180886693823, 0.01722127337291625, 0.044156284591386985, 0.000870270549233657, -0.15307819460612118, -0.00910733970849167, -0.08300475489333896, -0.5052829348682913, 0.20983317956977607, -0.1504734708247, -0.24433674606159345, -0.575168481484776, 0.037405064656347305, -0.1773499834703133, -0.3898117268148269, -0.18657745964501896, -0.10171168793290858, -0.6830314286817593, -0.37730346481226024, -0.15459433524240557, -0.5894219460143306, -1.074771964345394, -0.8681095379765401, -0.7386360634988185, -0.6924697203610848, -0.05498979776280412, -0.6262024868875956, -0.8086729111960361, -0.2554775342114813, -1.5332936456914172, -0.5680688291680596, -1.2671284707068862, -2.095520468718357, -0.5684209107036331, -1.402067560237762, -0.1591571493825509],
[1.3479567070943645, 1.8510638112477666, -2.1114182424747914, 1.2282545412410903, 0.6166506948184542, -1.0186747682541237, -0.9841168317101755, 0.016677234516000842, -1.0828035924185182, -2.726640816300126, 0.10968351715013999, -1.1394555099593446, -0.4981418431944771, 0.9859309141087523, 0.44829273368766837, -2.0359689202322717, -2.891569977052745, 1.4719073437867596, -1.7835871864673394, 2.234655107786234, -1.0973381787608203, -2.3242635087382495, -3.2841081373428183, -2.942109788854853, 1.671142315163165, -4.245932746749881, 1.5617224257857596, -0.5327298459865649, 0.3187610544835749, -2.849428189794561, 0.5273377311944273, -2.3563859108540752, -1.244465630559768, -1.2121696030437705, -1.108385229799972, -0.5997532535877523, -2.8417429456202465, -2.0174843747604574, 3.728418393324872, -2.4995817317946125, -2.1027364984159997, 1.204462171764829],
[-0.1295643139143188, -0.25180061016237076, 0.026840510348988048, 0.13838341357167042, -0.12405347333991444, 0.2013282899331681, -0.09716242657635013, -0.07280803791794475, -0.014863069992612526, -0.11865372078696443, -0.24336553886008766, -0.15302369060264107, 0.10656395110674288, -0.05621477319975135, -0.2423097874623608, -0.3559222655999977, 0.14223073270464587, -0.2619700169734253, -0.2838749817946618, -0.12770546626216808, -0.32874210418155375, -0.6080402579027839, -0.6013720311227521, -0.28219070637489957, -0.38574900522523703, -0.2854615595180963, -0.6638265429129936, -0.43243882008880186, -0.7087786525653952, -0.12262915566151401, -0.5621094360157723, -0.3724138415686157, -0.6407257262394876, -0.5778177668515952, -0.7266046689027658, -1.2406645463411237, -0.9524782619887271, -0.6746268533482955, -1.280321845783999, -1.1070170609005066, -0.3329616241429913, -1.783898796301015],
[0.11162850230497263, -0.05449108318477213, -0.12922902474650722, 0.08422075286457456, -0.22744180672149644, -0.007426259024856869, -0.09753103962690125, -0.08044050296016995, -0.02807570840451791, -0.35551601937832106, -0.08090172968843751, -0.07028401547088785, 0.04307560009712795, -0.14193233442966838, -0.142358290976249, -0.2012584494886126, -0.4526758891174422, -0.2174381500663088, -0.17532912059438727, 0.00430504479978592, -0.25830492751326195, -0.503619466526678, -0.3743153796690629, -0.5495332612513585, -0.2967792613271799, -0.3266537937545055, -0.4663728041700735, -0.3022435262812729, -0.8171176432481916, -0.41371595745446366, -0.4435264044826714, -0.3626288371701176, -0.327280459128235, -0.4303595047873503, -0.696226350566816, -2.7063159634866825, -2.2237074904987777, -2.725507108037872, -2.4978790069407606, -2.6038027940249253, -2.4669811243539908, -2.547735445515097],
[2.504825467542655, 3.023491892647893, -6.516706799293794, 1.5454100508070128, -0.5963897791733077, 5.05241544403807, 3.8011052532394634, 1.5358870275607956, 2.159729962158379, -2.554674824170125, -0.1269066524010994, -1.5281269270470104, -0.8589672885538933, 2.312986446318819, 0.8029074762517946, 1.7806577260952337, -1.0540505910930174, 0.08413781324486401, -0.5287196549628961, 1.7127819958436823, -1.1507048112248057, -1.394338117977701, -1.5694905758706852, -0.8559533654472561, 0.30198003057635775, -1.7130280955191064, -0.6165527022849858, -0.5279456113580896, -2.5498162006951817, -1.02814751166856, 3.23730114066587, -0.8982440921411645, -0.1988022053273346, -0.9414410031507958, -1.1548181490397091, -1.6347291432994147, -0.6450117006132604, -0.5349722529263835, -2.7520471173955063, -3.2505543158922188, -2.725226084709791, -2.8167323260588653],
[0.015419922931126984, -0.15544013727702655, 0.11279006456389523, -0.09250728287615666, 0.039169552892454496, 0.026600202615570213, 0.07774512868519907, -0.0647726510374398, -0.09509955845769605, -0.01595044162057733, -0.17310533682210394, 0.06511359764893872, -0.098001904028129, -0.05001218809867577, -0.23304940272832295, -0.32887536417343466, -0.07863544550839188, -0.23686693938640774, -0.22614763818398645, -0.21954920822402166, 0.05728173136499846, -0.036097689129852104, -0.16819960068272904, -0.28889594120831114, -0.5399330260771711, -0.2654353678057644, -0.08079548016695852, -0.17269040703214653, -0.6500012214338394, -0.3336460716643911, -0.5412259145262648, -0.29390472206143053, -0.4535906870639372, -0.40175966011515396, -0.3044216707868784, -1.183537413295506, -0.9321273625991636, -1.04238336633393, -1.835337147422614, -1.0809913382405374, -0.9851136056559762, -1.2415878722344842],
[0.770693672512661, -0.10401224898324386, 0.2714154594842774, 0.006486542301748897, -0.2575769129976731, -0.37541941972645343, 0.047643877141208595, 0.231027232783408, -0.07643677765855654, 0.34199193382059007, -0.07907432250433084, -0.34037974020175055, -0.3687609720497615, 0.5861937162913892, 0.560722183959698, -0.41586604786103876, 0.7529364752145488, -0.03387308168309807, -0.7364879791674745, -0.5955351236469415, -0.22452427634187233, -0.0004150275544944748, -0.3745992861261964, 0.6298534301985824, -0.9852956696561015, 0.8487104133121676, -1.3353458823177458, -0.10553837715154392, -0.759004540284062, -0.21084499868211795, -0.5527103484629836, -0.897946807366775, -0.16729525597864686, -1.4711764148585214, 0.4639026895704411, -1.6306416582022631, -1.7013698418271619, -0.0071242963621723455, -2.134916412462979, -1.6538744242587167, 0.3654516629826482, 0.5397886996832779],
[-0.08751524029562868, -0.3010810041882434, 0.4497326375015934, 0.7343446013109449, 0.43674283444090356, 0.34612781350222216, -0.3738354962015462, -0.11422019465202603, -0.32189699823364537, 0.26033748179873, -0.11358336541892472, 0.1433027734987571, 0.2684602319123297, -0.37220951049117934, -0.12127675278860756, -0.38844294518237155, 0.21634978882500694, 0.25239513301005934, 0.00018003205158074158, 0.1998946159738116, -0.2191631992521583, -0.3197285552920452, -0.03505675295151288, -0.09507987987678362, -0.10494880208680407, -0.581302740600808, 0.1644873849867466, 0.13038292644475338, -0.7960231756519697, -0.36350073521290127, -0.6091530789178572, -0.5087636815245408, -0.43907584877041145, -0.04885773973019426, -0.6311766450547672, -1.3001669479279625, -1.2821060098130077, -1.5142831843609494, -1.9206778702331182, -1.8252711482294424, -1.5464115778520557, -1.1242386626254726],
[-0.12676241531943927, -1.1191460705407192, -1.33728288971172, 0.6490213759171809, 1.9518898574994576, 0.9621541750044927, 0.43709013767604177, -0.24594739873835167, -0.48685347901687026, -0.9940997873808428, 0.07177662007213446, 0.6311573962342285, -0.060780913245674, 0.8432632583942097, -0.28012292685400236, -0.5516258974152084, 0.04029969913227122, 0.04061643223100416, 0.3485986979879081, -0.016814961728772903, -0.201119567249301, -1.1128974308603394, -0.9356856211413996, -0.534291170425841, -0.3749270718842345, -0.3846548689884324, -0.4960256930772719, -0.0092301028647348, 0.8394844761908515, -0.3916723120271596, 1.0484654160958162, -0.7936772537703982, -1.1096004383643832, -1.2793729419606046, -0.9035881521844644, -1.4285497321736007, -1.0424859387027463, -2.115298914252901, 0.6891229616186682, -3.435637919743872, -2.6132929102586537, -3.2060593980342436],
[-0.25079222240097676, 0.11897081732524822, -0.19648575817255165, -0.1405876074881539, 0.23917196854295575, 0.1406829755862927, 0.021399369246183474, -0.2912922797049103, -0.10757242157242407, -0.29715910106166793, -0.14762273493716915, -0.038043973200942764, -0.017811246160154764, -0.07881815004266593, -0.039722419663399545, -0.11166290824735722, -0.2341950991844951, -0.26483305043112804, -0.1699106986301444, -0.10019356191705077, -0.017299066818414557, -0.3141120451974891, -0.22780068555818103, -0.24888131904250455, -0.38518187280644556, -0.3035142838722223, -0.17705287051556934, -0.1916690379845137, -0.5618902396857175, -0.5551803029068172, -0.446999187058596, -0.08203255336201863, -0.4064334171524562, -0.6055295761790243, -0.7505981269943315, -1.9945412918718, -1.8941868224207081, -1.9803924560884505, -2.48469653957628, -2.4445766513669653, -1.9221233262685853, -2.378473149620148],
[0.17130853390816903, -0.060582788180205925, -0.19811539212963353, -0.3057409908114371, 0.02544576032060415, -0.07005962967691892, -0.22910413315894954, -0.15768076616375803, 0.027973285055699997, 0.0903604429904149, -0.3052042126623598, -0.030768448987846912, 0.01794043877113637, -0.1445335982121427, -0.15019156852716783, -0.24630592911405072, -0.12059872535654806, -0.32253808178965143, -0.24500990011699222, -0.30306398577246924, -0.10757824614328833, -0.3830401185059492, -0.2739609439290813, -0.30405595429151827, -0.4001441624200487, -0.20436309214188547, 0.031636509497434115, -0.11548386767697377, -0.8438324519163896, -0.46604438953589644, -0.660774988419981, -0.07790997770455023, -0.5493257789177451, -0.40460658480935086, -0.7578191901689455, -2.5337805726968554, -2.2456856026977667, -2.2808217654579934, -3.067498027391849, -2.684325790432583, -2.232936530061086, -2.5094822955820066],
[-0.5575264146288065, 0.8664206520195119, 2.165466120219601, -0.23543671439948785, -0.9745587457010101, -0.6539489648869895, 0.9410183530248967, -0.10803500159940937, -0.06935620884253249, 1.0901959040545164, -0.3602234511815126, -0.8896417903044085, -0.7413435570555073, 0.006049482750384318, -0.3576737349506474, -0.5001040953929661, 1.209465182689857, -0.8058593136195622, -0.6849630928249784, -0.15397466015725403, -0.11938299391097451, -0.01475186911514138, -0.952912921332502, 0.061004205134039084, -1.010697361529318, -0.7225506371694901, -0.11140288584807834, 0.3912663175452522, -1.0001395269975792, -1.9197109449937615, -0.6735328886287086, 0.5343350220879917, -0.18329820113455506, -0.5549932358339819, -0.7775133251744154, -3.2238894254214765, -2.949443844904154, -2.981821679361744, -3.9172613331107593, -2.642220166016715, -2.7496521755352625, -2.7809246176084614],
[-0.21881233403939832, 0.2115221612120584, -0.00767514450002479, 0.02924071631769136, 0.3158806882499726, 0.22160290545738082, 0.10652502371733956, -0.22466993052099352, -0.1611960383160778, -0.3141872875975008, -0.0163168244467225, -0.06437054887135578, 0.11878773948607764, 0.1107915244930391, -0.4106193761472203, 0.23844995820589948, -0.3321523302543906, -0.1544704972492812, -0.13473553389559206, -0.28029695440887903, -0.028045898888308704, -0.1876140537741546, 0.03736619512654668, -0.5103514706559316, -0.3548082483811509, -0.41988596638403825, -0.2641918907898408, -0.24099026704378265, -0.9669637206544891, -0.18417593396552948, -0.6529457861048165, -0.3247584181670291, -0.5160502738350409, -0.6740018491063906, -0.8590028433386125, -1.4172622433504627, -1.731481028821185, -2.29628972177208, -1.881963925566943, -2.5325525749751057, -1.7534641606526082, -2.307958330856875],
[0.06542491203867923, -0.6576721945557111, 0.8741125389267648, 0.48902058029920337, -0.09460321384946521, 0.5323767030633743, 0.14004244754393508, 0.17543345690497267, -0.32270203315442325, 0.5188752542108422, -0.06038430564473857, -0.06463495872293742, 0.3207877576173293, 0.17443878086969114, 0.29212909509659657, -0.06101878536409165, 0.1746747116411132, -0.3140665770682368, -0.023984566712023415, -0.11110413554403013, 0.007322957090619532, 0.10230894576473173, -0.22345264396471293, -0.15542750360224464, -0.12902858842927906, -0.13287012404727708, -0.4900827048598215, -0.3039898350835402, -0.4724518880379046, -0.1745096583616689, -0.5171056187410141, -0.1656731566773112, -0.17120825850502075, -0.3647657780404667, -0.5313190270842433, -1.2444713958811648, -0.6865148152326951, -1.2892761600222828, -1.7290776632699598, -0.8816974465249692, -0.968636538501159, -0.7254265450141628],
[0.3106868616836474, -0.196415859622959, -0.6267374494983798, 0.03202373314476346, -0.31954520924725277, 0.688893396101471, -0.25478652787332723, 0.2170697375632487, -0.14613580109018073, -0.279678274302803, -0.40299408805906767, -0.06309353600522806, 0.4251846867668318, -0.2817527053993157, 0.01169784978020753, -0.21116142545752584, -0.2551630009683355, -0.06025082227976725, -0.38023944933917797, 0.2898684886618371, -0.2946861254845581, -0.19427707802558875, -0.3853524708139743, -0.5564617144643691, -0.00506043474337324, -0.49023101039708217, 0.010690668916258902, -0.05471831419292206, -0.3591925029795019, -0.4130282368102847, 0.2569728532235989, -0.5070785788820785, -0.4621661295186348, -0.7410915023000489, -0.5302261586362406, -1.8091687279912023, -1.3959784176445453, -1.1255889701745712, -1.3900894207459604, -1.8436924397441812, -1.8085190165840366, -0.9716933520940154],
[0.2702458177173488, -0.6301607730600469, 0.4489555669997414, 0.6754683371894057, 0.29468079153456245, 0.3034318666050687, 0.007046393855351821, 0.38645683808366493, -0.4906988537084066, 0.19228794290316595, 0.12776030120363846, 0.36202723803153186, 0.16594613329732205, 0.2572157153610317, 0.29381229391331193, -0.029598404042426116, 0.0686238742947629, 0.21178329545347915, -0.0777525640801308, 0.14920335002241017, -0.10974549934740846, 0.014487370718749074, -0.47763043753813744, -0.04693829442987868, -0.4726855461925921, -0.22368005462642124, -0.18865921195483398, -0.31036222536844077, -0.5647534017267958, 0.024073049987195803, -0.49380887959319736, -0.5228669916034503, -0.2927307822181733, -0.22316412304101416, -0.5348127716352693, -1.2403408303325252, -0.936486246194048, -1.221595422411258, -1.417609380368732, -1.560229443519466, -1.2364345337477545, -1.3210504160663754],
[-2.7865128106657036, 2.9174841816366017, -0.04642190979595713, 1.71693553787226, -2.596438437082425, -1.9290876352337514, 7.288904428964901, -1.3382253593056208, 0.7580261553310939, -1.2556181238475583, 2.0408798062974154, -1.5157247103356444, -1.0157823012695457, 2.9237181296452723, -3.119658695735249, 1.4041082303197228, -0.44433190580541776, 0.049412759901043296, -0.4837909142565302, -0.019001866897903, 1.7055743956347362, 1.5101233682419857, 0.3854065585618159, 0.28109510360823936, -0.0979250905553407, -0.5419750764279349, -1.6142919930030575, -1.0990773967885885, 4.600988469561021, 0.1503413613925975, -0.17320860398572596, -0.7406799877640228, -1.314902442448378, -0.272922674278732, -1.181885140718168, 0.40598485660083783, -0.7283131139157609, -0.1202503280278982, -2.968440776906975, 0.9931967911545803, -1.3607024380523332, -2.5578702241452547],
[-0.19665405333413422, -0.31980542107619864, 0.1490955071589639, -0.5795075220488922, -0.32698504069531853, 0.296110934069452, -0.20396388636158713, -0.19577013408290297, -0.0032971885173064906, 0.05686747276657215, -0.9012628835341538, -0.154117712136529, 0.8403988521442045, 0.13554239839838442, -0.17531118042113558, -0.009179237317074403, -0.19283556274476135, -0.10603000752077216, -0.3578976199725433, 0.4890710819494253, -1.3350969756964997, -0.25832241079120416, 0.1989139513954744, -0.6750985300684199, 0.9396320910218368, -1.1807605125645575, 0.6538154779318057, 1.1199488160771798, -1.1332852314216058, 0.004132797118390316, -1.2978549344611177, 0.5043347813223645, -1.5605486340311412, -0.21683772968061063, 1.0860906222072493, -1.4953161686949434, -0.9970411645272331, -1.7103862804580712, -1.7161824747935348, -1.7716295409726515, -1.8740991660024295, 0.7663753112805225],
[0.29549323278400375, 0.01171531187195369, 0.37457617122539283, -0.23389619871237, -0.006790877630933169, -0.09223522073239629, 0.06181994585265666, 0.013420381422057422, -0.4480456566090479, 0.06221999561687576, -0.22748410532800573, -0.05115501748386531, -0.1769703977328226, -0.25656780572114374, 0.1718882989308781, -0.33736363493716026, -0.10972883173868211, -0.2851288736777983, -0.07145848086599238, -0.21405667033363318, -0.10132865957551822, -0.1854935279773466, -0.5646977081096388, -0.43272628858448325, -0.30515228969410213, -0.5516016295510147, -0.31681468595313345, -0.2548656806112417, -0.8621189858040466, -0.46004722778424245, -0.6807883400890777, -0.22514521398943038, -0.3515571837251321, -0.3350938046979895, -0.3527027368341997, -2.127626161188563, -1.628255283958867, -1.5842676394160438, -2.1150733043921814, -1.5887922461352344, -1.585608746783932, -1.5505228183369895],
[-0.14655844587394853, 0.24458487522760525, -0.0769299405471108, -0.25561470919397977, -0.10256594436476156, 0.024385544561567336, -0.2107965888917489, -0.22598116457189216, 0.05012181258388202, -0.0636600479581005, -0.17527117815402493, -0.3381951467677002, -0.01762446909113148, 0.031027685067476676, -0.03326473322924235, -0.11220469429239578, -0.08719124895792339, -0.1759579936017748, -0.18173608822920867, 0.04229358083118558, -0.13540588530153813, -0.18412699243642047, -0.26413088142817603, -0.11626318398089559, -0.427240350601403, -0.3676437061220718, -0.18406072238302743, -0.40537348267942297, -0.7878887113712988, -0.5530407714332448, -0.6020803598388217, -0.3519015012560876, -0.322663806548191, -0.7320225697107142, -0.7069747993222927, -2.2372005642255375, -1.97597966835698, -2.4624216703094297, -2.807436575897918, -2.474081067688865, -2.1753411658880317, -2.0679846896741747],
[0.7169394194411874, 0.13923881815295905, -1.2141904108527093, -1.0360502539126564, -0.35116983458752715, 0.34872827493669994, -0.20948608392817972, -0.2981072050250316, 0.004196117322046308, -0.7913677871463012, -0.16529239151890165, -0.533016068640789, -0.033903506832719976, -0.5724260805630864, 0.2909907966659764, -0.8886171034936079, 0.37550561074342537, -1.1072183532204078, -1.3943612516228512, -0.6112133246609206, -0.9172888395741928, -0.027417106502452132, -1.5808214508401972, -1.024883643936716, 0.6139384723148541, -1.9956572513945354, -1.3933801030462478, -0.40798138684835833, -0.8456491071919383, -1.8081829931682785, 0.044461760734265596, 0.3036230648853919, -1.650173160277084, -2.218761816269357, -1.237536011515815, -3.167925887300578, -2.822213753671106, -3.4028492861972817, -3.0747073553045747, -3.2176958602543024, -3.398055562698001, -2.7631815840418916],
[0.5160452814099861, -0.042468740390556196, -0.004173774029409005, -0.04512172563137592, 0.48433516843915964, -0.3320028182358733, -0.15339559806347364, 0.20349950866096828, -0.4330228502968814, -0.10273920436182869, 0.2770359887804321, 0.04253316334380667, -0.2735214419225651, -0.34946218331784545, 0.10785475463719835, -0.31073902577402907, 0.04751170923070697, 0.06522158610798891, -0.1331461233725706, 0.05863619675184083, 0.23211808543970588, -0.17582124193957413, -0.2826954253117365, -0.09445901344449133, -0.49116637000524177, -0.5823261500817316, -0.3288940355867549, -0.2360984524406449, -0.5453095612860236, -0.08850904350802745, -0.6745544944716402, -0.5014555815759647, -0.9530161838238029, -0.32855454463808187, -0.26728051817343984, -2.0563537686492284, -1.599606935648906, -1.700309448005109, -2.0871518760780967, -1.9205986358477238, -1.3583097738300631, -2.0774521764520295],
[0.2696543409801525, -0.1103037406713191, -0.04414703262228328, 0.7612156114942918, 0.659006575158394, 0.6633601177408709, -0.7380404295335274, -0.03359228935734337, -0.26022311774628437, 0.15493847155254237, 0.5140176328118203, 0.24017550300646107, 0.4483502223319963, -0.4759024005899674, -0.14224528451472418, -0.20970439266438792, -0.10368537765636743, 0.268643258520273, 0.023812146538806894, 0.04977283841509387, -0.29432591433377897, -0.01779601535757367, -0.2483742562325502, -0.12262077042886495, -0.007824821992426883, -0.28034291070515877, 0.4105339728154395, -0.30552798079616517, -0.5409323761462902, -0.19469339211700942, -0.6223406253893805, -0.7559456641627177, -0.49687831972568397, 0.20232743168013784, -0.7529564475734198, -2.2867072753482414, -1.6977135256191573, -2.357301503524571, -2.5549356680859607, -2.3705533220091635, -1.7523881067698095, -2.0127238693165386],
[0.025170935422676906, -0.12776391344826485, 0.04015892884701663, -0.1937204475461952, 0.0908987680539061, -0.10325831114089241, -0.17983906337607902, -0.0010433527752716982, -0.03232530734430222, 0.039237006108162845, -0.40496094236810587, -0.0452045524611411, -0.04155154626362935, 0.08394902487078572, -0.23859640202647175, -0.20229125366838088, 0.09722727241489577, -0.12392439064910679, -0.1543899401955538, -0.32959062418233703, 0.06785081978448768, -0.21534434543924338, -0.2285259304066706, -0.13717018226019756, -0.23431815091048402, -0.2180473553953417, -0.4005260005353066, -0.15858996481895501, -0.8853627576134236, -0.4278228985025149, -0.5626430021598663, -0.14803154584163047, -0.5249971916681913, -0.41704518344380337, -0.9249249940666636, -3.3757236968639965, -3.000404127483582, -3.327473375273509, -3.4666035825427692, -3.389402280344644, -3.1534500260380547, -3.330155358611297],
[0.012334132985288754, -0.2328336504610504, 0.19557733226636528, 0.1919575076378125, 0.02868440620788767, 0.009836390088012665, -0.012937966700938148, 0.03708003037029625, -0.21002699182654824, 0.1293549102007254, 0.06034364538903509, 0.10010832707415583, -0.06024246712295103, -0.08076670418532593, -0.14734334160402862, -0.14118268936440337, -0.25476508702784645, -0.04875744999930335, -0.059673125194420404, -0.05095100882287223, -0.16480925403797247, -0.44185554484896866, -0.03019871438324581, -0.23536472575852013, -0.1856959923260207, -0.5119424487107068, -0.3547191665650716, -0.43772248301779554, -0.8007405867403614, 0.3598654120438344, -0.591713501117622, -0.4529321023109895, -0.3735882046541311, -0.1880081150175929, -0.2594594500324528, -0.642537090805131, -0.8606674649855157, -1.4209674525374743, -1.5984294628467746, -1.4097057264239639, -1.391234533173599, -1.0523735427708873],
[-0.03230976626248373, 0.04740918967600583, 0.08653322559041987, -0.10294503437796075, 0.034946200541301195, 0.0005302754307818399, 0.34145240765637225, 0.057854851786309905, 0.09152063167110268, 0.09094234164849135, 0.0213600500834685, 0.18738106562513862, -0.11480522384592702, 0.21776145985561973, 0.2790826988534948, 0.42225192179857735, 0.24778568478375762, -0.2508359366401294, 0.4637945313501675, 0.22920896213244923, 0.09775067557163919, 0.2818357152980592, 0.036637552367956155, 0.2517268877919037, 0.16609494986493809, 0.28568820532113626, -0.14162161198756906, -0.2531770022401883, 0.4050667736026931, 0.12479777978565829, 0.514037672531018, 0.21044577314773966, 0.7104445687065616, -0.19230428181725479, -0.2690778616707053, -0.21604479483769237, 0.5400032593126507, -0.16231961752889204, 0.4971154689357309, 0.40211554051195375, -0.1880061495249472, -1.1832669808521494],
[-0.10818481971780705, -0.011502865869374903, 0.2575614152617602, -0.1240288872635478, -0.04516197520320013, -0.001333929553777463, 0.08747852211533387, -0.07930528873230012, -0.05995885578490058, -0.01584758189329047, -0.23061862192488425, -0.2093450000162061, -0.07552151378743996, 0.08786434095096199, -0.042266037887003595, -0.1261843797695658, -0.04399681830696516, -0.22676727727664484, -0.006402986442341262, -0.08221725739252632, -0.17934702473928757, -0.18772754321745994, -0.24771601668575655, -0.043867013453174036, -0.3826839388966425, -0.28857825467873444, -0.3216359334680838, -0.2014591145489557, -0.8854932250621661, -0.4480099911129872, -0.5161756917578535, -0.36760736367307295, -0.4382854614069151, -0.6096207467909246, -0.8372365078012222, -2.3945083900935256, -2.1133100168217713, -2.484070537131572, -2.8797813711809073, -2.530882322812929, -2.2960682241620844, -2.4502970504809416],
[1.2361050792366473, 0.44530286137218583, -0.35363047682431453, 1.1627817438523, -0.1965115348826878, -0.40689005686545365, -0.22608787541828054, 0.7459293863012735, -0.05379470370608561, -0.23163747959418363, 0.3037369739337864, -0.29997053040939237, -0.5870925091539844, -0.4314499537684046, 0.04017244214927485, -0.1510197415975227, -0.24532416048205286, 0.3312948241311646, -0.2008282402175759, -0.13490327768382848, -0.3038227651437167, -0.6731098349462787, -0.5367581807930448, -0.06460327378247081, -0.1316181915669515, -0.7055408063071821, -0.42366185419804525, -0.3488526597787998, -0.912887286718892, -0.5297232008861225, -0.4318740267346569, -0.6604015123149298, -0.11487421194994869, -0.22855022752214663, -0.7480160139864555, -3.1409386449881866, -2.614538043885291, -2.9803123055863345, -2.5345049000531596, -2.9520086728493338, -2.7326448620138324, -2.693275563047242],
[-0.02321502141249586, 0.07431376715522074, -0.31520083604405597, 0.11201098777625125, 0.44033190909445247, 0.3152728891173755, -0.17113620589288378, -0.11224311417945713, 0.039807447326327385, -0.01616712828901257, 0.11797204348084095, 0.1634613915873031, 0.3294518153713343, 0.053147188677673415, 0.11885040993747803, 0.1475200281984121, 0.4527080757718222, 0.039022752391883936, 0.2644007076988218, 0.17428177391313418, 0.08036605072957395, 0.1947199529551566, -0.04856936178653841, 0.6673285680268085, 0.32207819287105377, 0.2733828420030193, -0.07055268598848942, -0.26201789126567315, 0.23962332109045215, 0.3823868080122547, 0.8430926296783238, 0.3247475509225306, -0.10064615061041754, -0.21678589861672634, -0.36472798077850593, 0.22626944038245556, -0.15380314365354888, 0.9321723485754906, -0.9497231012079658, -0.5656763445045416, -0.34567350430383176, -0.6886674613786448],
[0.5892050019932744, -0.003914090114298237, -0.06501427779968134, 1.096443462675169, -0.6606973696163547, -0.7854652179713306, 3.32265158138615, -0.015990752422052723, -0.5068664983477176, -1.6262380496743392, 0.6415075647686121, -0.6859332238722793, 0.09391372991695614, 2.3348193354243723, -0.22625610054637155, 0.021488453362338265, -0.4457849519801288, -0.32369051570663504, -1.0000769356896426, 1.0007053867399078, 0.9282907804812037, -1.0702931833680789, -1.1067215935662744, -1.062696943964411, -1.2412576003043956, 0.3605823517750038, -0.1275298630504919, -0.5405689500497184, -0.444047470200217, -0.5151925788486843, -0.12240249123076394, -1.230197326932445, 0.4518646966293991, -0.88195658188655, -0.9995349731062337, -1.4040855115131454, -1.827002147295345, -2.0072637015181556, -1.4225045696961423, -0.6855740980053688, -1.8678792526077759, -2.5824458649390776],
[0.07848412394013936, -0.002110621147882173, -0.13415669928361618, -0.10896053225802958, 0.06706438881790278, 0.05657312410608547, -0.06906599323807423, 0.014770376834125121, -0.2523077310658839, -0.15788624795833303, -0.18189924286977757, -0.19505176748307282, -0.2766400783223931, -0.0010228473191932319, -0.02276595776704428, -0.19491784300044301, -0.22686573833433785, -0.39492043123951803, -0.150858958042565, -0.3564418252352274, -0.19272852763687623, -0.3782096917447158, -0.22494157350219843, -0.3324144068043275, -0.3836735117588065, -0.05643331356961531, -0.34128014391490236, -0.3411890038164858, -0.6153253575378509, -0.5167069133075509, -0.43660830806135836, -0.32063440992815406, -0.46092734187399187, -0.3935742254156609, -0.7275351067455444, -1.4514513624069256, -1.2525722589703816, -1.5014142508844797, -1.772966968571951, -1.6593385439997843, -1.478915051898443, -1.9119015715350407],
[0.5921509656286736, -0.16170928737305537, 1.5422482048802386, 0.4500408362997015, -2.0702231830521267, -1.426381150776971, 3.227555915706385, 0.6078623285527107, 0.32158029822529205, 0.7903964830027109, -0.35976494485424515, -1.383555546019524, -0.5367432165006915, 2.17186244725904, 0.08745792583172642, 0.6586834624598468, 0.3443215471244312, -0.9470574476184707, -1.353514878403267, -0.09360224431462091, 0.3049895731145022, -0.31684263292242304, -0.4805534692127991, -0.9749791381023857, -0.47725339906729747, -0.7968894040270923, -0.8721085822779742, -0.33101683465978, -0.5520623735864649, -0.8102208313577096, -1.186384949314596, -0.6545472732144759, 1.0790561806223549, -1.1046070551090013, -0.014567359389011143, -2.4386538347647018, -2.065070764302698, -2.228525496328283, -2.8177375099058986, -1.1611524205366988, -1.286144159307703, -0.4245128145748147],
[-0.24824075941775417, -0.7321113376563161, 1.5829071599139177, 2.330771305192028, -3.756529730796256, 6.231050077103939, 0.021515866494493385, 0.33938450124899866, -2.1289748921170872, -0.3479635034392541, 1.0804940430503092, 0.3992849348658551, 1.7950747453869642, 1.3638040825709834, -1.5895481740152002, -1.0214605213409975, 0.7068444024789935, 2.128540689415948, 0.29549850377177533, -0.41283390542014836, -0.963836769294475, 0.8089957553578652, -0.5074851441362621, -0.10697003904348171, 0.3918471975593875, -0.32462745682281163, 0.8955196139580812, 2.5105006917356505, 0.10763759827256589, -1.7934138390231278, 0.7469270450187017, 0.01627128789348396, 2.3828634983558388, -3.921116720304023, -0.38840340804862256, 0.04034142346658496, -2.461924300477805, -2.534635044604597, -0.7294331455314008, -1.8559642933279492, -4.978474790265767, -1.7884586626790984],
[0.21098982152894274, -0.26392109840320604, -0.5000398676975732, -0.34709573682641537, -0.06937158089403732, 0.14431783201253007, 0.05426171596716834, 0.0777722108781787, -0.2662425591371217, -0.34940688647862517, -0.38052433778946876, -0.1344922346124856, 0.09022829318140689, -0.096830376485605, -0.24166271275446025, -0.17816194721572676, -0.4240603778447199, -0.3197772972330699, -0.006353121117243876, -0.25626148855763276, -0.3545904438897095, -0.19714340989060358, -0.33270110011992643, -0.48176837659208016, -0.3183236861374576, 0.11083613613936232, -0.17016325724628192, -0.36807881010596516, -0.7042594521370692, -0.5947980527689392, -0.17843595335577572, -0.3807394236150423, -0.30275744940450994, -1.0147145986033443, -0.8311598210686602, -2.474692892308697, -2.167572155097639, -2.3283932326145496, -1.9908231940128431, -2.774375031440262, -2.461047198957322, -2.978227138970594],
[0.09723961304991083, -0.07701111051211681, 0.08643433917500133, 0.1982813894074231, 0.2504557295645061, 0.08093373671709354, 0.03051038780297766, 0.13170832992653378, 0.024589777967802477, 0.022852882095405078, -0.14114811399325955, 0.07947054330695731, -0.004871986614070024, 0.06229661637176089, 0.1682900300709809, -0.05421902086325039, -0.33536816911021267, -0.19978762010817744, 0.10644923164124735, -0.10818090722013507, -0.05490434944603286, -0.3610530746917873, -0.4494208021079361, -0.19686860295236042, -0.29592833751411185, -0.24903783438606614, -0.2372919126631211, -0.38477276690381096, -0.6223587435301203, -0.1343967243940047, -0.6168910912974488, -0.608870200322844, -0.2384376305286786, -0.04336985341592366, -0.4728721516243064, -1.9812754412723945, -1.4077094629772784, -1.6750538999557347, -2.136297684112554, -1.5301330324379843, -1.531812274657718, -1.7216281436939107],
[0.16288509592483885, -0.6048470829827962, -0.6375553291055122, -2.525991648133727, -0.2764419939448663, 1.34089645929428, 0.2009278637120778, 0.6763793967595213, -1.2593012510511588, -0.6360276209179976, -3.556533444366871, 0.8841846453016478, 2.2748916770888763, 0.46220972965380475, 0.8813994655991068, 1.6698595600348405, 0.3015938352503988, -2.7389449558551533, -0.29393173730682787, 0.9694311129405474, -0.6830149249839561, 1.1058417556173552, 2.5692362874188284, -1.9973682389174128, -0.608793129759564, -1.9496945976842364, -0.5420807014437314, 0.12868068042292888, -0.04067146954377024, 1.6861223815336244, -1.7981834988598009, 3.3133386680728134, -0.8651681033756301, -1.6632825906142028, -0.6281287212559186, -1.0970055958653617, -0.37800822012400964, -3.2878939094016406, -0.9916517768426615, -1.111793713826146, -1.1943628633389078, -1.0998090227622936],
[-0.12938744802978197, 0.004825326398847968, -0.05953859677304469, -0.060815856427248995, -0.04851156699827646, -0.11027464000715623, 0.002135326306234289, -0.17104484685191945, -0.20987836950949104, -0.2497526495129107, -0.2789890217950906, -0.16360079636328123, -0.015267355454183049, -0.1602295041231459, -0.21925839500072442, -0.1879611530452783, -0.30351293205939683, 0.04869676747849898, 0.008749232146409795, -0.21328420719040728, -0.224340775109703, -0.1263230241099447, -0.5099146497974306, -0.23576335397570142, -0.05349878342540369, -0.36546268561046463, -0.4679004467897562, -0.11345718517506995, 0.08700379148155571, -0.4267797183348937, -0.0025847494108277146, -0.15904320146101203, -0.15864459964656394, -0.48341228767960726, -0.34997150819420003, -0.9109643697196497, -0.43113874615818715, -0.14599823287404112, 0.08715718241585999, -0.5654075769097927, -0.5995488474216927, -0.984002484483477],
[-0.11735357922883616, 0.0383030380069932, -0.07649775587049071, -0.20462615182872215, 0.0005909583958362097, 0.2746689206341783, -0.027426454430890573, 0.0538577729775755, -0.27392515061768785, -0.35979620870127105, -0.2944373623061038, -0.13260474373681277, 0.08517733612261262, 0.04846926938563818, -0.3271505018812343, -0.1807003280221878, -0.3686555118428069, -0.5486474022313849, -0.12232549810643915, -0.18172995799054442, -0.12950917912002663, -0.08415042111437701, 0.0018831896192685839, -0.5225044874933494, -0.42646224339216243, -0.5796248071691616, -0.36777367252266674, -0.2637747339750696, -0.5900175555290135, -0.7153105379410414, -0.6615840527837736, -0.18445858073970264, -0.5863434132167807, -0.8385877302929188, -0.7897929634052508, -2.8860059310148327, -2.4902792257918946, -2.6947556964184596, -2.9602297303611196, -2.8900489674499577, -2.5299747002570316, -2.7059204685887797],
[0.009165285967854871, -0.14662917066644726, -0.02669476997333429, -0.23663900767756754, 0.48736694606954095, -0.20552450189839427, -0.04799622165851068, -0.15784674957969133, -0.13665828612967656, -0.040568780925439155, -0.3208939673279612, 0.15383131236400935, -0.19202100669080707, -0.10714418810971249, -0.11533224706246994, -0.21711421465797195, -0.27071730512271813, -0.24843066574020475, 0.07204667402980387, -0.06601725286947654, 0.017972959145787462, -0.11839868488412549, -0.29858749796754547, -0.3779032175590407, -0.37289573588982716, -0.022151888983640528, -0.18053594241611, -0.29533958800195553, -0.49160331011136166, -0.34257314677198525, -0.32953557089713975, -0.3974592084285706, -0.4835750636195353, -0.45493648190100977, -0.4382091206629727, -1.1697360406820014, -1.0335596446741335, -1.0591893517703392, -1.38629651225298, -0.9775575338503709, -0.9350418990559298, -1.6351391579146344],
[-0.6879199213937995, 0.8099863044964184, 0.9445999042463991, 0.2974772186492334, 0.9303431416553498, 0.867819792639027, -0.10680460490857258, -0.18973480974608353, 0.4524219374442999, 0.5075886932514884, 0.08889687108208624, 0.2597947110208893, 0.2774173356722446, -0.023797523608582206, 0.13262013769345377, 0.26407923421870005, -0.11434046855590725, -0.2180359847666116, 0.32489062584344686, 0.0600471428613073, -0.004793519699001884, -0.1616663675924113, 0.529526867677655, -0.025373691573353153, -0.3440476561239314, -0.2933447746309606, -0.28912852741118233, -0.03077652348345338, -0.9963442197046435, 0.5872416502897598, -1.1369467654844085, -0.8472547247867122, -0.4901030517649658, -0.3357185838272285, -0.7050975693912992, -2.0631671445604454, -1.684830181024174, -2.726741889826106, -2.869910179184111, -2.5717207763804084, -2.4012900349295263, -2.3269933650764427],
[-0.024214569374960394, -0.2947300838375349, -0.0038905919139751634, -0.17136654332060142, 0.05191606876804656, -0.14603161104864462, -0.07176742449436682, -0.14146112214713658, -0.13994315151837333, 0.050564098709308614, -0.25434827019762474, -0.12915632890248793, -0.12797202912854075, -0.2387419433428378, -0.23202656392130372, -0.3611320370899064, 0.0310663921548957, -0.25169234453305545, -0.049522312997033927, -0.2256000028177155, -0.28457297802141107, -0.2387453709156573, -0.27605318318748895, -0.1652542693102384, -0.6067317479113709, -0.2509319077692336, -0.27822550315744743, -0.470103534241803, -0.6848560441594522, -0.3950189604118649, -0.5006650461495876, -0.3104050097783844, -0.4363008080850815, -0.44422573875482324, -0.641215945064803, -1.8017559048814134, -1.3746849252512747, -1.573694189600858, -1.9645129392788998, -1.6222361973777097, -1.482218365334876, -1.8492634316651293],
[-0.13405643425260652, -0.1514969635617994, -0.09326582588819735, -0.1309386713857912, 0.02200317681691004, -0.06390531175186465, -0.07497417029137868, -0.13002848850293633, -0.03832136589493423, -0.18358086733167278, -0.028986055628670144, -0.2206245222702739, -0.013942348310246917, -0.10862104635492766, -0.2683713649459242, -0.19415876243617908, -0.2516083894846842, -0.33275810396970545, -0.43464914382475556, -0.08939196842223009, 0.21756432016957664, -0.1821485264126925, -0.40322388521964153, -0.38243709169260726, -0.35621875531067787, -0.33018780650784885, -0.19745956690508984, -0.06700851870387374, -0.7834382660548294, -0.3266661738721384, -0.5354699876269124, -0.34938592172859373, -0.5823326484620666, -0.1622824681700076, -0.6904359473592316, -1.8287989272413696, -1.4719542378906285, -1.8333841541858655, -1.843534061942341, -1.6864261835078593, -1.397021859892559, -1.729869801995165],
[0.21888893183484368, -0.003251370883993492, 0.07929156353165498, 0.33216574337626825, 0.0917845604597854, -0.180759721945945, -0.2533037401537404, 0.1181946957704316, 0.022585424988380207, 0.07980961297534456, 0.05516188026632965, -0.17147059283598176, -0.22184086937255942, -0.12174823088660253, -0.10569054375010521, 0.15677605245174206, -0.006454492370741542, 0.019008422101890837, -0.11369988000640086, -0.11579823770494095, -0.20276995009072435, -0.531022898720646, -0.5330923195413481, -0.10352687707593226, -0.09224525906485836, -0.5271206923692161, -0.5220345677317564, -0.15259081975445327, -0.599274866162987, -0.4226387205320665, -0.4120708561568477, -0.588839211374412, -0.6261647772078393, 0.015843495217767206, -0.2646721492899315, -1.9084689126380878, -1.4001790350441352, -1.402458122216957, -1.5306931352007886, -1.2919354303398967, -1.0807999364015286, -1.1029939769117196],
[0.8674700773818164, -0.06825049041051287, 0.20066701067718318, -0.5340499082647906, 0.04530398843868153, 0.23974666880010775, -0.46932590668613444, 0.22131034285249584, 0.17316727605879814, -0.44792394273766023, 0.5332251326174923, 0.2860402186432821, 0.23892495927491683, -0.5499714458363913, 0.354312666842072, 1.403699012315916, -0.6637357755016576, -1.5610879358824203, -0.38861035855250636, -0.5997953259541239, -0.648122647443284, 0.22646238150520814, 0.7480982931789606, -0.14010708388103535, -0.38014186335505906, -1.0575976632247837, -0.371296656004181, 0.11959898698248247, -0.3788252683439844, -0.19829016252273546, -1.2848552011556724, 0.6626423804008894, -1.1877025298808186, -0.4746404573184427, -0.2349138012904488, -1.6770124394301378, -1.0361858272038822, -3.229586737773154, -1.9713679607001595, -1.2792786184592548, -1.4555166027623103, -1.792475228342507],
[0.04053617289957361, -0.41726768578770024, 0.06828704918827712, -0.3277327262201459, -0.21781058776071252, 0.2061684582527765, 0.15650995722070346, 0.01642091317831315, -0.4338004708476368, 0.16531123457475433, -0.5092722147932435, -0.047801743745398, 0.033986580972802674, 0.2113037337856126, -0.049959369553104906, -0.579003518510821, -0.0629815488683231, -0.4196528886559566, 0.17870640923825912, -0.19017332507310847, -0.07817957360114239, -0.3851573487580332, -0.1119597525127342, 0.20654791821832838, -0.5038659904144703, 0.4325071297231642, -0.09201553152114221, 0.1849045013277176, -0.5714519829157846, -0.7289785543043126, 0.0785713213456424, -0.20182891856536847, 0.23741221861688766, -0.3819729592463902, 0.8982705912386612, -0.8887890161493924, 0.18316195553949405, -0.313828985391215, -1.660222197388773, -0.08527710756314355, -0.35869434980825465, -0.1788023869105718],
[-0.15565314771576536, 0.07085322993345623, -0.00019881409784206974, 0.006418403569775105, -0.1500261585568077, -0.0534198437514119, -0.01561043899968022, -0.10633003717133743, -0.05634352807971404, -0.41779176562509646, -0.3061885618017398, -0.17131128395394965, -0.1155382209722844, -0.18717745179241296, -0.11393586127545721, -0.3055758598964943, -0.21677862632362926, -0.3182325033061489, -0.2790997000910325, -0.3972196044333284, -0.13731746895650165, -0.10485653248018541, -0.13767638214961345, -0.6605586136034295, -0.5494649004156369, 0.03512683103107068, -0.20070467069583003, -0.2486517456850728, -0.187999698939483, -0.6894488293548588, -0.08328352377479696, 0.2173314286054916, -0.43516799404809337, -0.6066871098886831, -0.6345592290003226, -1.2078380810257752, -0.8551908088936966, -0.8128406703634353, -1.4684301911139608, -0.9464277406525775, -0.5979532070910663, -1.5021759213092318],
[-0.4442636392058883, 0.048718637649822316, 0.15738080666780424, -0.08340905620649562, 0.30260543184291816, 0.18255254630556986, 0.17877038296815015, -0.3191660673933714, 0.05369606044612964, -0.3143829662622905, -0.08899348888235996, 0.04803337913640883, -0.031551084637702155, 0.10441201480610525, 0.021735351683697803, 0.008507990618958575, -0.1611622560684709, 0.08459421606506093, 0.05954169803840434, -0.15898017325090114, -0.05514904325550634, -0.16631244957097763, 0.03347792452709337, -0.131135024527517, -0.28287347426456516, -0.13420918699841777, -0.5217295413053894, -0.21179230380126252, -0.8028697539365119, -0.34091168320356624, -0.1865483360606712, -0.2032766120284095, -0.5048551473535422, -0.45321322020881366, -0.8590429516422572, -3.322753301436267, -3.1461970185132158, -3.5682358440573405, -3.3481443562289455, -3.613701128160455, -3.3204729313864974, -3.661796962720846],
[0.048993648657083104, 0.17601366515842598, -0.6118337340082064, -0.3779372981137498, 0.4140795353208356, 0.41321052174892137, -0.15030241999850102, -0.0832306469303961, -0.054319559115614834, -0.730588769119601, -0.3210352598850052, 0.23383186214722732, 0.20915905897099102, 0.07650492537735516, 0.07873142776418317, -0.15518492353112212, -0.40284871772781133, -0.29078953786870537, 0.2779883121714837, -0.20265528799913535, -0.20869770548395697, -0.23225798517805224, 0.31482552029844824, -0.35773473381845367, -0.5982764764968439, -0.032976080694491705, -0.5641064473286724, -0.21634785392892475, 0.6049677480911729, -0.12558547010890272, 0.1671589711205656, -0.4163262287508568, -0.4711827670750883, -0.8013959151702156, -0.8217502818696574, -0.5639319739560151, -1.468267510734569, -2.0631326406656165, -0.6076072882931397, -2.2452299823043003, -1.4862578884952589, -2.3639213732400473],
[-0.12942616337178137, 0.28968688255524344, 0.0810092046046525, 0.0432080352030919, -0.061530367852081186, -0.07905747801306076, -0.05730908603880328, -0.169077742672984, 0.14525323843476562, 0.10047108020715642, 0.10836836217187686, 0.02183050142837334, -0.05720687638600162, -0.2931817258309844, -0.09634568194799024, -0.25250879032466206, -0.042501319338216104, 0.010584490076325557, -0.01950644221755729, -0.29813803179266735, -0.2192107756111721, -0.09968079397213621, -0.2784466252848698, -0.13908836957181694, -0.3515167314317302, -0.2359777693748503, -0.3228856591734861, -0.10697083077968633, -0.7799472750503812, -0.44958328567183364, -0.43245938789187366, -0.3875771277119253, -0.5839933019959348, -0.2654494251065606, -0.6317724168498454, -2.047445250150002, -1.777602389731101, -2.2540529227103177, -2.5553260509850886, -1.9768197281106101, -1.8624154666618093, -1.9827869615213838],
[0.1645568275358293, 0.21164871625409357, -0.2485417799307295, 0.5550462791083858, -0.38519890358785513, -0.09242633760684198, -0.11895486538937462, -0.0028919977430726263, -0.15726664635724524, -0.7440680737841431, -0.07160860588838676, -0.4607074638283515, -0.05266382337451863, -0.1344906157651206, -0.19996506070947823, -0.36957026600248816, -0.8166048430232253, 0.4811463279243865, -0.4752239497537076, -0.37070818356621993, -0.06820493739692944, -0.40783273384277174, -0.5841749665979173, -0.43356322900342753, 0.17335918830283037, -0.49254508613623366, -0.3855954268903809, -0.40639044517552736, -0.7267588396627808, -0.7584320156320505, -0.5081039635343298, -0.35761553821423, -0.24065938660090583, -0.624216959390353, -0.86951250460512, -2.956553465725128, -2.6637176970663767, -3.0013626861849434, -2.611583389490704, -3.330005029629171, -2.934463264386667, -3.107219709675934],
[-0.3410576889193585, 0.18249356651625948, -0.14184510000588899, 0.1101890112663508, -0.15667984971497723, 0.01803201420315679, -0.3820548515380104, -0.1720691721557626, -0.6519821854063634, -0.37186841116723107, 0.6126241647418784, -0.25847372511355576, -0.0420390274131535, -0.13569540782949405, -0.1980889987422961, 0.6403132512510019, -0.6780630685912835, 0.5103571183483088, -0.5874281663966033, -0.17972484627395413, -0.1254860680742475, -0.731875465965504, -0.39797342731815816, -0.17836858967390537, -0.4847518164036026, -0.6418870344742962, -0.673270427369012, -0.5040594242200891, -1.1771441019632296, 0.05165145860742872, -0.8962420413332801, -0.817782488958278, -0.7438736636366852, -0.33618585136203255, -0.685032197229983, 0.06435677835726683, -0.9511170614134021, -2.4651145180615552, 0.21892212871401992, -2.600451015952507, -1.823608415161544, -2.007567805172617],
[-0.22666842491576183, -0.05951154886748382, 0.15212571259271182, -0.2524277190275078, -0.02903964017557712, -0.0168016608877118, -0.2484283723881361, -0.24503253357758137, -0.013261547608720873, 0.024864392413076173, -0.01474818585693613, -0.2093633040682579, 0.14659537127048458, -0.26083323032858274, -0.0008475487080286366, -0.0382113446328609, 0.03207489886746087, -0.07266403329958819, -0.012663291613003417, -0.0062684378316525515, -0.02484285937803884, -0.10102593374071969, -0.06451677934643565, -0.1881287746677397, -0.3440623272445906, -0.30456548467141825, -0.152106987197137, -0.21800212943792244, -0.5683168371166568, -0.18974137882353506, -0.4644094302425658, -0.13525265381792284, -0.14600263372455652, -0.41900860063111633, -0.08270240078787648, -0.5716961815993828, -0.3488609190455125, -0.40208144625109044, -2.073983319331384, -0.6867819053768472, -0.5367843102119565, 0.025978186188341103],
[0.2642615734858167, 0.3721788294454494, -3.233072510932101, -0.7377881939294045, 2.5454086957083115, 4.777697246043359, -2.3890628405317, 0.08139843059408264, 0.6191331255160792, -0.8642046833979312, 1.3251803186405462, 1.4546416387768104, 0.2571523583319422, -0.557797199908841, -0.5321620653030473, -1.2099071870050753, 0.13935608053652046, 0.030484447790189682, 0.9830188477372724, 0.14270951409113436, -1.4586339445181902, -1.4002772602300524, 0.7604465816241546, 0.13006998802764919, -0.4124858138209726, -0.00171801307699663, -1.359459682806101, 1.7147917890564581, -0.9319291389754616, -0.8359982005296955, 0.7076921767946549, -1.2649269382960653, -0.9304751741489223, -4.9583766055288105, 1.3174006292468003, -2.1298476987668233, -1.8713721239606604, -2.034504176292881, -1.6677371648537922, -3.02670588234408, -3.8800632751399102, -2.3943680382028014],
[0.06248415947100943, -0.06452279513924444, 0.052973278636762024, 0.015771945851051516, 0.094178363087352, -0.22640116168873878, 0.10722021463845434, -0.1915369975293625, -0.05509379134455637, 0.14971622243587576, -0.26337011819287093, 0.14650676627525702, -0.17507917761445693, -0.029950365694570085, -0.05760486334918765, -0.3157041275078146, 0.19835835183096978, 0.04921681980705804, -0.06644113067632505, -0.345037409150769, -0.1312028507721881, -0.17005556052898693, -0.15633070909633748, -0.19305205061234143, -0.3504709261303969, -0.15755011420602358, -0.22452181961835802, -0.3424421436109577, -0.772572623616224, -0.4119261772212109, -0.5454600151421242, -0.3747094242349235, -0.35722855129320263, -0.2536985323223988, -0.7202011815780995, -2.0674627215364363, -1.752041223158598, -2.1355388169955716, -2.5953326670818617, -1.9128303757588698, -1.918260575659651, -2.1581883133844486],
[-0.011204776407512454, -0.09563042110101319, -0.01592203320839956, 0.24053521301622696, 0.2945227505850689, 0.1539453583648369, -0.04148068516409427, 0.08254679487607432, -0.2485701792876226, -0.13546392581136535, 0.07092779194857844, 0.009649003426607414, -0.08634846828184803, -0.15417495424117442, 0.04212426841754224, 0.0602973578873427, -0.1556860636115706, 0.008519595201791809, -0.06409233131202759, 0.04017716354164767, -0.12470367812665215, -0.2629762834311112, -0.20768871124698235, -0.2113321802075258, -0.2039521117947862, -0.3811303291823602, -0.3315812049163087, -0.15783436982287802, -0.6754288120300257, -0.42867585788200896, -0.5246133507496175, -0.403208451714398, -0.6828882204311237, -0.39936419905963666, -0.7205601506227414, -1.587459381163515, -1.5647644182195029, -1.9331419454306098, -1.726103932165129, -2.037579771339593, -1.4549310981061063, -2.0747911101574834],
[0.4793863931644333, -0.22803674005484953, -0.598915919652991, 0.14310572247821343, 0.41239591598695646, 0.45566308491853835, 0.03972716218909889, 0.33120778939061163, -0.2748067363213692, -0.60440080775689, -0.13050349672386055, 0.43506725037628285, -0.04673866558389061, -0.3295485547779411, -0.10142041352678177, -0.12589488835260593, -0.4381863557188393, -0.1985661704679863, 0.1797800143294612, -0.042831957408137764, -0.2981884096604067, -0.20760779853527694, -0.42696574503316687, -0.7036894105240101, -0.06295040336070923, -0.16766986057430153, -0.40140374498830644, -0.3358843689241059, -0.36174322808108106, -0.5878941050343506, -0.3197029810307757, -0.8310637239106461, -0.4439616632825529, -0.5022559403210998, -0.7890101181465364, -2.554212217695801, -2.1112526529212894, -2.3899262264661196, -2.3558139777069327, -2.645474982016129, -2.07743204609114, -2.583810429122895],
[-0.20231607020602788, 0.18320048822731386, -0.27424044292259875, -0.07673597157888531, -0.11341372965873356, 0.09390952046071388, -0.14078729866525488, -0.04045098421528417, -0.07644113236584077, -0.10969348357672642, -0.13271048609583347, -0.08481223482260274, -0.16712544694012743, -0.10272753443694312, 0.01144257561323996, -0.21515440296544214, -0.3588499881509247, -0.06706562426109716, -0.07388119673336792, -0.14634080702027877, -0.16485178193835412, 0.13830318442193917, -0.24138711436599877, -0.532854013950355, -0.42555732606036717, -0.4833515119273385, -0.2305450356965183, -0.34213895917510934, -0.31630067699944364, -0.5110131502140023, -0.10077069708682647, -0.3859556911286196, -0.4275246924392143, -0.5326896096746525, -0.25513216182004933, -0.5377294678039092, -1.213382362742448, -0.8710631997860706, -0.9016908422024993, -1.0284093846262947, -1.091541795713371, -0.856484744378389],
[-0.05486659604349217, -0.1000200119883735, -0.1275513741676643, -0.23409017998856224, -0.27307627075986324, 0.15440031084089062, 0.09482905205904714, -0.1213727542316487, -0.020971038239315672, -0.4197140591312412, -0.2665865426055771, -0.1178254463545992, 0.05102190360248902, -0.09192465292256533, -0.17999203371852943, -0.10543962431818145, -0.08663991497331817, -0.41374421988037585, -0.31016334433986137, -0.0972797483181999, -0.3647929425816127, -0.14230967369117384, -0.6124747056222846, -0.012401384085080614, -0.3784618154501793, -0.5317141402597648, -0.3459028452607705, -0.23181276606820345, -0.38849677815160605, -0.6215888874292675, -0.257822202232753, -0.06321980715486022, -0.24339917801463634, -0.7760878475679032, -0.6816673885345182, -1.6038132585276705, -1.1436062914475538, -0.9679672458531863, -1.6832118974649488, -1.3946444339063575, -0.815372515742982, -1.3332032521742012],
[-0.2914252338565335, -0.004303570625104549, -0.250057933901103, -0.33833198213071575, 0.03722437511365963, 0.2078046632546172, 0.019588306725815026, -0.09921751197490657, 0.024179455652709538, -0.14816999083560722, -0.13739704607379039, 0.10113196336209951, 0.07835333361805775, -0.034820050162008134, -0.23064804691164725, -0.05454379882432892, -0.04747068972272741, -0.30252345037811945, 0.15096226376718083, -0.023880856682933656, -0.07089648404415863, -0.024319072446379746, -0.12448127464647847, -0.10209740037886202, -0.32584922560477775, 0.026777723955914424, -0.25663067786381144, -0.2804494211015646, -0.7170114927900871, -0.3269965200985967, -0.4839117643344161, -0.14732986367823375, -0.5849672284195875, -0.7249125038930609, -0.7159984032286122, -1.534153582988061, -1.5530684726873736, -1.792876374968993, -2.478633781339916, -1.9796361225816794, -1.7454848757163273, -2.0817088331117266],
[0.16897091671734035, -0.23042741484960264, -0.0694323888150255, -0.21422608152981848, -0.21700659500095365, -0.11359313256003693, 0.13215904074692603, -0.20530880880524988, -0.1414335097474967, -0.22759718934089931, -0.45407547665174613, -0.2674301923666692, -0.12130000710482282, -0.13873434730518794, 0.06325570283057204, -0.03322863520065524, -0.3168666346436702, -0.5245465107226098, -0.17179303457567974, -0.1938051298377351, -0.1613446667191555, -0.07657258703619167, -0.18895889901715268, -0.033955737067481055, -0.4788103395669397, -0.24415044271047967, -0.3154752703591714, -0.214116746701569, -0.6043263245810125, -0.620543956680521, -0.42747090155048806, -0.3647006637939856, -0.38650528058693834, -0.6897003461524596, -0.5563183610561314, -2.4112533611899325, -2.03339531335632, -2.295648296186395, -2.4241375640272116, -2.2835192851767676, -2.177178418349539, -2.420945197794368],
[-0.31454606363084564, -0.3657816633002574, -0.498702460869793, -0.5879760436842587, -0.29478907528913284, 0.12611891971871914, -0.07027055072181002, -1.1497337602140068, -0.8518828093631903, -0.8604701346935093, -0.7033016469837509, -0.19950438768545112, 0.018690832749532028, -0.07034037483341786, -0.7302625464397191, 1.3710690275754014, -0.5515774391376796, 0.8956322935165186, -0.2558170055551289, -0.37077673755327534, -0.35958624146867557, 0.039391571754255617, 0.6568975379348817, -0.1893710923163832, -0.8838294194672743, -0.8151289859751115, -0.3610508375804058, -0.34268866526772473, -2.510286930114454, 1.5519953213285735, -1.2354367758012987, -0.12488183452430066, 0.25438797632583393, 0.24006938729354438, -0.6588619097996528, 1.4317323259168582, 0.5822176376260865, -1.2498406230463979, -2.200433445304635, -2.188555633997189, -1.0622854995891124, -0.8093337978788336],
[-0.06238734383727175, -0.03138946867703862, 0.03342118432065514, -0.046104875634280326, -0.16321310059646088, -0.019377580999718518, 0.19174625521792327, -0.01981890099339983, -0.12518428904962303, -0.3748278398276819, -0.07616782996170586, -0.2183550289742564, -0.05513717524929036, 0.13620300028503057, -0.4714181877950919, -0.19574664065210245, -0.5553233235642713, -0.16383654852075583, -0.4039431038959571, -0.1581034104606447, -0.19644345102765465, -0.05135665460819961, -0.36627243987391256, -0.342317781200594, -0.2949438806610403, -0.2380652198009894, -0.3448000891256153, -0.171273281416782, -0.12771332268586222, -0.6194920024000534, -0.026595474136098186, -0.4507164423207672, 0.007403025207519751, -0.6181858994769157, -0.5538835810392586, -1.1073742063342193, -0.816263829333722, -0.8231018109547167, -0.9411129346238871, -0.30524672673332937, -0.9026111225079806, -1.1578657306814595],
[-0.269450428131127, -0.044666989917402186, -0.34037371055859833, -0.1882131684102853, -0.09239139308168443, 0.180240369268792, 0.4482095704379949, -0.04479705209158868, -0.03389684533627775, -0.39364630445612764, -0.2110395599075825, -0.14326818967198962, 0.2541982630811065, 0.23483474175762042, -0.6328064910647524, -0.4008447352323912, -0.2991158061479284, 0.06172839221572085, -0.2811550087192292, -0.045818777330894844, -0.10656348097262672, 0.1569437080024554, -0.14142357367630712, -0.0056969090808194536, -0.5319730944464145, 0.057944353407351996, -0.03859524916691405, -0.3162376617782031, 0.010404403101319964, -0.7398037601046994, 0.12943555080459343, -0.33660970918180627, 0.11837899053484309, -0.5134470171632132, -0.291512093262646, -0.3613254189835862, -0.5439130136213903, 0.08873888812305164, -0.7615463139306008, 0.11487555690568203, -0.22241840934609747, -0.6842609452200575],
[0.1551177628478855, -0.16458431984425867, -0.0795198377703692, 0.002684565802673643, -0.018450982178780634, -0.02866331246398549, -0.15983080097322214, -0.18661689087232503, -0.06658336659864827, -0.11640893947479762, -0.15102252130425867, -0.22229925278396453, -0.1721953784074239, -0.18555349128088444, -0.193826881001375, -0.2410432721110694, -0.1128351385510327, -0.14323469404559844, -0.3103913679680124, -0.3766148913833733, -0.2960000317956998, -0.2582817123703716, -0.5793992364462756, -0.4430532546008614, -0.34569633352696766, -0.5421470646200935, -0.20256418641322907, -0.38117453395838363, -0.7165079878083277, -0.627791503707851, -0.6612961288085544, -0.35323487151959215, -0.5863491982133776, -0.5439760122602451, -0.6682955314230503, -2.132488640837545, -1.7032900992088988, -2.0145202664038315, -2.1119079355963177, -2.2475135882411816, -1.793471558734712, -2.055189007138003],
[0.022747515444180508, -1.14396518864877, -1.3428375509360448, 0.11727189464664033, 1.0795571631397223, -0.46586285235576685, 0.2775418279707735, 0.16474949815369785, -0.09202690000453466, -1.2001968228925548, -0.9411781667531401, 1.8110649092328182, -0.692958277807473, -0.16804636907590398, -0.2436070159250226, -1.4661167117105802, 0.22041159020260578, 0.6856338360193657, 2.892517733674116, -1.1029123467323543, -0.2947506803293447, -2.210142491971868, 0.40925481582825735, 1.1603318390871271, 0.11700361262841336, 1.134040696301647, -1.295658947761718, -0.6993059188195965, -2.0075308200148037, -0.5618440501838128, 1.3177821645937287, 0.07682739150826083, 0.7281264721617864, -0.9074667435896471, 0.027690026053012257, -2.3347877006456024, -2.0565465070309985, -0.22023356816821266, 0.3321113563622767, -1.1818178763723408, -3.0096678301587176, -2.2133467913177594],
[-0.9758980235977289, -3.0354466361279946, -1.6633571254088368, 2.200271207284549, 9.370523644265175, -5.138226162690197, -0.6345604269780811, 0.3022972595637566, -2.6717451387948064, -0.2311173872078633, 4.087099240400328, 4.439311177629539, -2.429351522227118, 0.9834880105130197, -0.06452676153350678, -1.3512989260664978, 1.8784596692852145, 0.3988495517733747, 1.8594892780069612, 0.8708571000093339, -2.05829879278165, 1.1987335597003907, 0.15978381116288734, -1.1576469011298853, -2.4307850683398082, -2.7814800889918128, 0.9874599795453329, -1.2069925284249157, 0.381837644892135, 0.2607959883299215, -0.731464678462036, -0.680059899356084, -5.090085431986473, 2.797886619566069, 0.5875365332438812, -1.7974397443812637, -1.3464778927850944, -1.9207356674029075, -2.4991697289623502, -4.884226023320265, -2.4337025564721166, -2.7803875717876774],
[-0.13201093373417294, 0.08990488850601108, 0.0723952277572632, -0.34423435165431054, -0.16117799488779025, -0.008686830960131165, 0.14835981388874506, 0.038616092141502426, 0.10840741557442234, -0.07070721103254728, -0.15920774781885652, -0.225793710571991, -0.1319079670327562, 0.05041265907183092, -0.1799470039347729, -0.08682624355135492, -0.05346146045607926, -0.37579850358460604, 0.011893955914155098, -0.07586213031162897, -0.0484087466975916, -0.09442740386198648, -0.06066623981374785, -0.35366162691140385, -0.5249183160084664, -0.17349224887357323, -0.2697797541201022, -0.32443373681584325, -0.6901644044092455, -0.5404136412597316, -0.40501141022506354, -0.1807312978180514, -0.3940049011024536, -0.5420447170502899, -0.6339385192043374, -1.7621437169283642, -1.4950060343482325, -1.7399655664099318, -2.328609266337959, -1.743655180127368, -1.5921643813236204, -1.934629182587305],
[0.3678433310131774, -0.45382319481144145, -0.1505902437224751, -0.6927380795682277, 0.10819271373498662, 0.32384839800155246, -0.03868847694654002, 0.07468541624419632, -0.3148828900000168, -0.21804099179632536, -0.5432634984585043, 0.10175217229308305, 0.38939698329954137, -0.17999399143056014, 0.18060778627200866, -0.6457465098715245, -0.07963603262886922, -0.7923071146718059, -0.563640224791791, 0.030739561857296333, -0.09240253118828919, -0.2691633553576383, -0.5087361972835842, -0.10683479303223642, -0.23660052154632003, -0.5951344061497047, -0.061298172041561926, -0.40192966256610463, -0.24760743179260172, -0.8495610446457768, -0.3080270773427703, -0.34892057421246514, -0.9505055555177809, -0.5246961592597528, -0.5471882844554897, -1.0580694640603252, -1.125204954704228, -1.3088093861667365, -1.7395140781167233, -1.4716429890552312, -0.29359881613340316, -1.486716553640722],
[0.09225602143983072, -0.15674682086664957, -0.17585293644021985, -0.30985502748943133, -0.11608922998705577, -0.05869000326678433, -0.0589731442144227, -0.21109540024046433, -0.0700175536670454, -0.1518871518598696, -0.12976305920339984, -0.3316812859343427, 0.049342737085076104, 0.016077289761428953, -0.08969988622795526, -0.21051491591185265, -0.21016919253802832, -0.35767862012842516, -0.20484776805779897, -0.17083998132062012, -0.32533019044053135, -0.1308399868771792, -0.28606773391055346, -0.025213254886406463, 0.05223961777619295, -0.3921584702382121, -0.03260846388784779, 0.21779511100630178, -0.4935046917788943, -0.6906056094106049, -0.37700792190630106, 0.3880365131601743, 0.2649528319912077, -0.6150453767590341, 0.0025465834278497923, -1.2347840947434083, -0.4328638386476419, 0.1543510625867289, -0.49462921561159184, 0.4595786111778214, -0.36604253296720063, 0.20315117447334777],
[-0.07995234508997401, -0.04364701557282157, -0.15854072763705412, -0.22735772595374368, -0.06491152027875667, -0.048636973016035034, -0.26014302895086283, -0.11152132593450094, -0.15320836090299736, -0.08712828667667241, -0.015837594232578397, 0.014524651986503094, -0.022231959983226975, -0.3721676927824507, 0.019083032993058018, -0.18533110968400876, -0.20119310815191316, -0.40366277950312995, -0.24364466571825172, -0.6081833150701165, -0.11890329142817199, -0.23854149539513678, -0.4337444760239626, -0.5037455252607933, -0.6277258320666946, -0.1272874081933842, -0.6653047231443207, -0.42323486620936607, -0.5841496557864587, -0.31485275414704395, -0.5907992246395056, -0.2845376857302286, -0.7036576331867154, -0.43803740852874706, -0.6524399318227251, -1.4035841306260737, -1.232662034751673, -1.1791676997019847, -1.7894090208273656, -1.1584124771520694, -1.2317383971066682, -1.4402141538080266],
[-0.0361861463940321, 0.16524513273315597, 0.044764370762532875, -0.18541657574794249, -0.027123394653919435, -0.041734609896747106, -0.10538229222569284, -0.16782878573836738, -0.017243604483846226, 0.039694548006021, -0.14164251170247144, 0.0927796580529788, -0.18799228864035128, -0.15932393842554274, 0.06731504910362929, -0.2616160518221281, -0.05189327448699458, -0.22878394249174117, -0.056844044692947245, -0.17362753147339646, -0.21526330604503893, -0.23908436573906602, -0.40976095683234176, -0.10240497510517167, -0.3305892005178168, -0.004304826863335106, -0.3793359717807508, -0.11217726361598684, -0.710041353559223, -0.41099589438498363, -0.6127018287545878, -0.11462585059176614, -0.49201320182654007, -0.3410791104253733, -0.7912465683515628, -2.3854756509116433, -1.9908791624382853, -2.2219319875040022, -2.743929944546006, -2.077567333723292, -2.0980724530966666, -2.316953017358143],
[0.06689512421302582, -0.32493469677930764, -0.04525882716717499, -0.19460721608460083, -0.1766851975144251, 0.10015905204331091, 0.1446372176463529, -0.08324871959752324, -0.4314441840105577, -0.07961764210422421, -0.5819008347970643, -0.14657622626574043, -0.031242530993929433, 0.0484880270823798, -0.1283303715625658, -0.07659568391955424, -0.048474738737040834, -0.14011191043554233, -0.4695017406676022, 0.0005912905907942654, 0.07716566850966548, -0.4521347939647734, -0.4018668819099469, -0.13435277988274888, -0.31732703147571045, -0.648389551927221, -0.37715192225835764, -0.2432600427503551, -0.6871069763179151, -0.3152636575022756, -0.10954121037118944, -0.2776142684204385, -0.5336308064135111, -0.5687681709170711, -0.6381595423183374, -1.5852606016449413, -0.9978543206919992, -1.4494698683116234, -1.7482310723355552, -1.3499721632822699, -1.6045934744244157, -1.8215376634135252],
[-0.051441710966833254, 0.260937712127785, -0.36009125502657097, -0.14325440740939555, -0.23410735095454785, -0.1423298102221778, -0.36532322435161985, -0.24262595378273658, 0.07770560534652485, -0.4383102191119863, -0.15162311620510985, -0.2247478767958425, -0.025082283436847125, -0.2510690511767354, 0.015351466869008484, -0.4267904922788371, -0.29084810666017913, -0.6537600661502605, 0.0788087979342347, -0.4191318017938623, -0.4134014312335427, -0.3487401386273093, 0.05796847165544999, -0.38193121092233745, -0.5466893902322022, -0.07945885435424627, -0.2592103452393773, -0.3742289759830806, -0.6366159243122379, -0.605454769766981, -0.85357062635491, -0.25969930088972865, -0.47167369688576477, -0.6773830824773942, -0.6889417948763068, -2.0238208509693383, -1.6683223992997365, -1.9308545516316633, -2.258209480716249, -1.8837447760608352, -1.8631277474419297, -2.029043253045087],
[-0.08270607760148319, -0.6584546349783464, 0.2992566192095406, -0.3797658407452005, 0.09890310684684815, 0.29470469552382167, -0.3502232330552351, -0.6095452699843423, -0.0318455826303162, 0.37732030874548056, -0.45474693276175127, 0.33871522086049094, 0.13358347059770748, -0.25754142542089864, -0.5200687298554312, 0.32396212939253166, 0.21433815018347874, -0.6632563294713657, 0.37733661513002664, -0.2764006588471298, 0.050312000970047954, -0.3033893865360314, -0.5263602721942693, 0.011640755092635135, -0.4243813042380017, -0.28160844133037777, -0.4034495421414799, -0.37212353387771224, -0.2886652124615866, -0.12096083556841258, -0.32902318188767876, -0.04067230302848225, -0.5164454254353389, -0.9980411800138628, -0.8146582852756048, -0.9249737476248743, -0.8824641351762613, -0.4244571910136298, -2.0212167911973076, -1.5323615828630586, -0.8217167116780602, -1.2864795211715512],
[0.2481197139461848, -0.21417007124666831, -0.0004024954815845505, 0.5295181166587799, 0.12644786298155186, 0.3200591979873386, 0.028991186645814793, 0.2629912264390641, -0.2278645606950375, -0.02223131627465093, 0.07459109094897, -0.13812000456576737, 0.18371684314469647, -0.07721693778587066, -0.12037198439625947, -0.3009574680298431, -0.15733040456383274, 0.13328349483128443, -0.21191364500404544, -0.07111112675438669, -0.14732634242584058, -0.4591047270296095, -0.6442297727020233, 0.05166568418687569, -0.17504932432666115, -0.13870736587215834, -0.5078272292781801, -0.11264888459657278, -0.17930016577757021, -0.19614544955215002, 0.1268595448163143, -0.17202168413295915, -0.0790245427958553, -0.4079507058186403, -0.08521659877260279, -0.898433584939286, -0.5690802304133241, 0.29738094654431335, -0.43334641007744223, -0.7013352893932469, -0.5350873628519607, -0.56389229669869],
[0.5184281686402692, -0.5632268570527033, 0.1863943156237835, 0.47163780095728286, -0.3438793148800591, 0.19051330217912704, -0.17162061714235163, 0.47040575670458384, -0.6055937063685679, 0.1527477425253678, 0.06524966617163876, -0.26129171032526455, -0.12587126901159834, -0.15319719479506116, -0.10245313640307659, -0.4431930345354048, 0.06162505966548307, -0.04636630210202972, -0.45545129560327635, -0.3621810771478761, 0.21953918808814218, -0.5298935178479067, -0.6795070212550584, 0.4147814740459514, -0.14167568065227779, -0.6248944496038225, -0.6211840547575201, -0.2330515655266016, -0.7513053798300403, -0.25859714179519744, -0.3375442500685458, -0.33641635807995535, -0.5247543925635417, -0.34989419414781153, -0.4501107172825401, -2.7331400682914486, -1.892281156879237, -0.5840083841398384, -1.8809294481145973, -1.129340451901545, -0.946937887408369, -0.9716140263884412],
[-0.10427276434371335, 0.1334905409073572, 0.715613036875259, 0.5477537895587811, 0.5026292275665842, 0.49182577626157864, -0.2985068061793842, -0.22517412351019325, -0.08327592174885907, 0.30898010127338055, 0.3379206316631177, -0.2186237275344243, 0.16828269660477566, -0.2862327823844508, -0.22696505778641402, 0.026372815301239957, -0.33938493716648366, -0.04990135003506895, 0.16584229636101427, -0.038962807856329836, -0.10831319301085082, -0.11229121882602361, 0.5868984972779847, -0.2347978079328366, -0.07743028291553758, -0.4611086723060079, -0.25790631614522513, -0.06969522705752482, -0.8187688435011038, 0.5482495302368913, -0.6427365029385498, -0.6466933770912747, -0.39065890102128226, -0.5151760968004918, -0.1304740332089904, -0.6176496869368161, -0.40882274773891425, -1.2086841471214336, -1.7128441591809929, -1.8086302234195923, -1.1845152996039532, -1.013036955508335],
[-0.012419996982885543, -0.14287920037151014, -0.0716664229161558, 0.2611421423153867, -0.11514320696361502, 0.1152900491149719, -0.034438658895869646, 0.11422878513856838, 0.000482888730747127, -0.10795467905956299, -0.09650526154559737, -0.2167711345937627, 0.06192351908885015, -0.2793680126037893, -0.18019504145180085, -0.1995752116612158, -0.08966406572645216, -0.0276363388115872, -0.17978815883913107, -0.17838445774497746, -0.2525451802224829, -0.024354171195172383, -0.3125868110298884, -0.1153685332020925, -0.35632566238428287, -0.35742871755131433, -0.041602644504486426, -0.05450232069346885, -0.4258796106121973, -0.5402474380402692, -0.354319822441031, -0.5127369480280064, -0.5354664564792334, -0.09095697491030118, -0.4771709324374564, -0.8682120026180663, -1.0615613380955202, -1.0159137410809433, -1.2549380193882897, -1.1244758052333914, -0.5279756593136695, -0.8962156287784272],
[-0.0723848847522808, 0.06641233911438756, 0.04930434625174776, -0.21650772706234359, -0.1703676424979954, -0.10503787070372603, 0.16021965567824448, -0.23016386693268992, -0.44048831394438426, -0.2103661869176538, -0.08750233626044646, 0.13508662205353678, -0.10455468478021988, 0.05288531424865169, -0.3433186480201114, -0.4728000508364285, -0.14724200163491624, -0.5078644108141905, -0.21583333850394024, -0.054690432816569504, -0.0642813861436432, -0.39932065652412074, -0.42728189150655566, 0.0645394268274885, -0.4637889785068553, -0.21905242409548473, -0.06469015669033217, -0.36016911602921703, -0.5923194039681391, -0.3107180698903396, -0.08980673057552123, -0.5789280531921424, -0.727616984815044, -0.4864312882986397, -0.6633417804116062, -1.0627130485647647, -0.970262106452442, -1.0276680178709539, -1.6370863275702434, -1.147864836644132, -0.6905693307757997, -1.5921341011600176],
[-0.12719821117594546, 0.11490008830742916, 0.9577393730107722, 0.5810296080893459, -0.08366865511625066, 0.9612234218677076, -0.6632490636367397, -0.06396009316193152, -0.36244118073283876, 0.04747594593338559, 0.5271096377305853, -0.1614410454773964, 0.47380688111321406, -0.19764774134838312, 0.07208845084449124, -0.19897207964508157, -0.10240635599460732, -0.1733828258871928, -0.056900327045063606, 0.15456879787469247, -0.26791858373343563, 0.24118002871418362, -0.09305046395632914, -0.5232194707718058, -0.229773084181286, -0.12554503874881834, -0.36862062998043116, 0.37840911409928735, -0.8534244978739701, -0.3119269669868391, -1.1199262146823368, -0.35274144202768665, -0.43958977240955954, -0.8434379372398149, -0.44958644614229604, -2.561827157747071, -2.121131802143317, -2.892151096525538, -3.0964429405722083, -2.5978118112034534, -2.780140805427117, -2.2501038327844984],
[-2.624532537340166, 5.591682049479869, 4.893506410561171, 1.0081321229916547, -1.461497408913772, 3.9896685513206798, -1.8858655137515694, -1.2586002135861096, 0.8383438048905758, 1.117023822493317, 1.3714991962772738, -1.4754757429735101, 1.6157851968559205, -0.8421758749219206, -1.0402045018726893, 0.7318730273934423, -1.03312451061301, -0.6022394518154098, 0.338032966456222, 0.51363532493175, -0.9166927251180678, 0.19633864799313552, -0.8329833745331404, -0.6192016957096722, 0.11616723884390583, -0.009008502412150754, 0.6045505373906775, 0.3725016665308917, -0.7440531180008414, -1.267154990705343, -2.1598305663147443, -1.035097603473432, -1.1700008432920401, -1.4909441905722203, -0.6711597391276971, -2.8645891604042353, -2.6702795870895257, -4.219015702971476, -4.487325729060702, -2.7022144432265724, -2.694346018507385, -2.406108974029815],
[0.2571391015642269, 0.07803684692750175, 0.12000312294428339, -0.1640763231155354, 0.10976845714684677, 0.014166904049968282, -0.11131891761362693, -0.060744695512813626, -0.3222611807006494, -0.050238415353094326, -0.2702870974861832, 0.031838926741609645, -0.41453820102918015, 0.17610881925165275, -0.06612394680230703, -0.10991309386499037, -0.32167118175266557, -0.382325441749077, -0.07970119703313812, -0.07480542023871278, -0.2658937795410817, -0.2782137152899054, -0.40586788907716237, -0.37696178098202504, -0.6763090710765686, -0.15762898729450314, -0.7076510610166064, -0.4284024872016087, -0.7030906110800341, -0.6135127930963002, -0.5318457946816486, -0.2487996186397036, -0.32436387401733596, -0.6942714538703331, -0.5904002114467096, -2.379079231257568, -1.5638315992054137, -2.0279335193614525, -2.428499134152136, -1.941482725881673, -1.8574377205980888, -2.413501377264764],
[0.05866479577399469, 0.08597549087217433, -0.09453566651379224, -0.3524529217916734, 0.0726598754969362, 0.0180351233677461, -0.17887153423727176, 0.02744614369205146, 0.028587239885925472, -0.08828743940321179, -0.19388909979276842, -0.18670740730129193, 0.08073006390528963, -0.01625453496782955, -0.026633520277458324, -0.207593782512937, -0.1789701722128715, -0.17436292090948963, -0.2153237364677581, -0.2876138797015895, -0.11725087218333109, -0.18035800877907107, -0.32356535846951506, -0.16378506580963909, -0.3891268962557885, -0.19796006194989665, -0.0996955694646626, -0.24029888706398506, -0.8531744842221285, -0.5244117647274985, -0.5461021982297319, -0.15857004354765894, -0.5049625973742249, -0.5280047903284101, -0.8444108609121938, -2.7095087873153454, -2.375403548132812, -2.619378847517232, -3.0543060596680878, -2.8485721197594374, -2.5063026569750515, -2.6935612618126354],
[0.6550284659910403, 0.24425761621183129, -1.3502725468274164, -0.016657159962363773, 0.8675921075629418, 0.8391049549733359, 0.39711595771681324, 0.23287169777134206, -0.2184153123866928, -1.2003587069983361, 0.14616878193794253, 0.19827686806156683, 0.14181388702882067, 0.30076736155115036, -0.05766580222433517, 0.00549975132937129, -0.5508243902280182, 0.04125405397388758, 0.08049409325391835, 0.01699609038898754, -0.08489099658992606, -0.6173346797712347, -0.6789238844832034, -0.5744685492024436, -0.42151608645829863, -0.24424798151111243, -0.510922068473707, -0.3622094501592785, -0.19648273546540904, -0.7027550546046452, 1.324412873954664, -0.4947607249701461, -0.6089448896376831, -0.7341139180320394, -0.7223231056151848, -0.94814028810437, -1.0880188110251976, -1.6183352313317458, 0.9594486786337711, -2.6666053542925394, -2.132597792343321, -2.628610648719728],
[-0.1749344536018687, 0.21877719306771792, -0.31986000079230664, -0.1587448229762807, -0.1629572788922897, -0.05789586740451724, 0.19581580604817597, -0.05881068714743901, 0.07390274292733028, -0.14640737663089734, -0.11723156580529417, -0.1782589568278412, 0.02413631015020087, 0.13385595547854254, 0.0028028887907674827, -0.2087466963680038, -0.20637640676289903, -0.08717437589762635, -0.11171234077132869, -0.15409482470913605, -0.07944580091199022, -0.21263304272049968, -0.3317959771045521, -0.1904606766463768, -0.4012372992420545, -0.33699442075563196, -0.2508157035815891, -0.11531697574956953, -0.7380764959742034, -0.5712030679599003, -0.5462973849812205, -0.28891388487516384, -0.34397413820041356, -0.5596765017044768, -0.7705520721519629, -2.2309714163338072, -1.9838313034034647, -2.4345189080011904, -2.631896898802657, -2.3979989860623276, -2.2493118857699534, -2.233675565122042],
[-0.21701442585662759, 0.11602705671744407, -0.04569066603216372, 0.007331470336114442, 0.19453016238900128, 0.13946215290841213, 0.23933456620457746, -0.13090121251117912, 0.07185399656554985, -0.12564293174266522, -0.0936206255822648, 0.0585387240890462, -0.08317874349450001, 0.09289127667701755, 0.026827006100356158, -0.22785469802453598, -0.0030227772195409747, -0.22265291320591557, -0.21452061234087758, -0.19357565229640117, -0.0633114948477068, -0.18197009668198957, 0.0009287841835130784, -0.05990479195622188, -0.32214117930882796, -0.0895175284314261, -0.3075536592492144, -0.30691587173106283, -0.7270509130304984, 0.3561226520856352, -0.34814505775214283, -0.39021948954339025, -0.2827590247868784, -0.4927912465289446, -0.3597792925959002, -0.2701276436085974, -0.15995656938750974, -0.8375928510576232, -1.3244460910171365, -0.6582445728857812, -0.9882927530810866, -1.2990897807942157],
[-0.15957427152604003, 0.2870872774619076, 0.2601020847814625, -0.041899018193596246, -0.2606189385096772, 0.0786008253721219, -0.09520117171273171, -0.14139590487639472, 0.044064012108593495, 0.16493977214470232, -0.07801982210349094, -0.29736985512606107, 0.01209570753607114, 0.07819595922965727, -0.029642521007805386, 0.020303558724322133, 0.08348736820027969, 0.013417759444703404, -0.06113359844977061, -0.03561348533975233, -0.2011582319913978, -0.13587205634733382, -0.2528834278971684, -0.1396600308582738, -0.22757536819985164, -0.22588666504172825, -0.42948414379224575, -0.14926933441482343, -0.7355983282730455, -0.4050053403521183, -0.6889370121361019, -0.3035211162112888, -0.260290538576497, -0.5635668628509961, -0.6358074374066652, -1.9384664620045, -1.7672860697276058, -2.006355153042759, -2.5502608619618052, -2.012337294552732, -1.8904517260823344, -1.859742724088788],
[-0.15674920239031856, -0.07870020993284335, -0.11691817160513482, -0.13798952478814588, 0.07417255894849337, 0.17057415437464143, 0.01214703949917992, -0.20399693469662086, -0.07381687848926485, 0.24738303116057275, -0.07631935280198807, 0.08457492524579234, -0.10088463175493237, -0.1886590428438134, -0.2976997797699585, -0.5292178312791, 0.4523925237149891, -0.20552247901928586, -0.32055476884650685, -0.4254630377919037, -0.10784103145885704, -0.49395086986130055, 0.13676665620894313, 0.24122158226604917, -0.34589475901559535, -0.025721260872095694, -0.39082958630161435, -0.4806160429481098, -1.1165768102786362, -0.12590447596166954, -0.5009047184036637, -0.5922704553703282, -0.5090460717836841, 0.1400334768782401, -0.7419033754975451, -0.6137176388279816, -0.09354011151590838, -0.9255040078091556, -1.2967407307940049, -0.8170705256649421, -1.0519205515716177, -1.545456958146928],
[0.17838582019777358, 0.08182521819369142, -0.1081307711528438, -0.132627053144558, 0.1566549113706885, 0.14653500131016478, -0.07993737541313622, 0.01229535544759765, -0.14604316194693903, -0.172150364148421, -0.29537360471538027, -0.10398695002837269, -0.10094233663159052, -0.2379441174055839, -0.11279466217532645, -0.08849300003803708, -0.26810809382670564, -0.15403284854586596, -0.09721317042991925, -0.14957393529622323, -0.007697118251521223, -0.39354393698799306, -0.2735489838142422, -0.08557327476788937, -0.19365440001396636, -0.2477945890931161, -0.47028153556741475, -0.19634212753324856, -0.7092649244735827, -0.3954444714265006, -0.3920090073050758, -0.3950281812435657, -0.5350112659611191, -0.5135832197425471, -0.8771051529568962, -2.829201279018126, -2.5392989369797494, -2.8422819941762563, -2.798509763338332, -2.9746176911106414, -2.6661160335477594, -2.98768815508316],
[0.026221285573926063, -0.24305876389808287, -0.2079582912511661, -0.2523367700886755, 0.2052971229119644, -0.2728263571394544, -0.11515166790980348, -0.07552797078211104, -0.42807759674138746, -0.198836735629109, -0.27564120953865695, 0.24193241334740814, -0.26876674152840785, -0.2515831205606077, -0.1802279101038682, -0.5052487683184614, -0.3158456954794313, -0.617785389045874, 0.041683756132307855, -0.5792196053794936, -0.44399105627251384, -0.10288016092283808, -0.5373146448590297, -0.10591958721814762, -0.7379576580648971, 0.15687042247404026, -0.03680793012008037, -0.33212823324591967, -0.3914647804916011, -0.7028739352533763, -0.3443224945716896, -0.3254333408779291, -0.3079013606494512, -0.43340327130948497, -0.8163490112963563, -1.7160594208883992, -1.3022295646101918, -1.2361580590207375, -1.7402346465470013, -1.2996398245891885, -0.9945146329935316, -1.987592603658657],
[0.7899380598575773, -6.861882017389881, 5.067736725571482, 0.5639185607713812, -3.6243996562146634, 0.873533345931249, 4.263563886416881, 1.6067853665113812, -3.554016204770893, 1.058848026592259, 2.1402366183435495, -1.3574086693869754, 0.481096873036584, 3.129433924575932, 0.9027507902578358, -2.3328778189415336, -0.888493827445629, 0.26032505322882, -0.07982822008560238, 0.8831718150003877, 0.7247602565912138, 0.1500619614335988, 1.351622092953315, -1.033259873240991, 0.8973156449005896, -1.36124782605022, -0.379140052059224, -1.6622322531841975, -0.8703126689628533, 0.015561922820349627, -4.4877659634502844, -0.37069806113860304, 1.2638408671065204, 0.37495425601889054, -1.8007319679028735, -2.8422784963562098, 1.0491263922507061, -6.956052753414728, -0.30919204731825434, 0.7914523013917799, -0.5504437314714106, -4.1219148927771005],
[0.10988546590401499, -0.06895670259049362, 0.03300900081323142, -0.3878243239808736, -0.021783730181472095, 0.10125898956758374, -0.0064329453555004865, -0.018342271910229538, -0.16190583679637802, -0.09830524315206167, -0.37298172503863536, -0.005004085587264112, -0.13068126814300873, -0.1720424941786157, -0.01294398001732066, -0.2671601422276286, -0.14765714979265893, -0.0992210539568659, 0.09675273387584843, -0.0468914015729592, -0.17762357679878, -0.20595135574823326, -0.23745936014344846, -0.10570425566197383, -0.46789257777135784, -0.180676521308931, -0.15455636421207053, -0.3321529740840977, -0.556700543992766, -0.3922341185244757, -0.4649711964313139, -0.0252750410328201, -0.35161858767373605, -0.5564215612354059, -0.6431183353515637, -1.7349630572391868, -1.524576875911142, -1.065558758305474, -2.303049391965729, -1.5035827691729649, -1.2857667424978945, -1.5405644397886646],
[-1.6270638827188686, 3.714831321974419, 2.6685277448440248, 0.6458005497133327, 2.1865189623550285, -1.300022950270943, -1.6529376077815512, -0.09210273181803573, 2.4390894800463685, 0.2433221767338234, -0.31282165299622894, 0.530594941089332, -0.5887349888973286, -1.551775254319519, 0.47909570557242404, 0.684709928638949, 0.6478000904641424, -0.09647910949684424, 0.1508240501479412, -1.1177891016333692, -0.473337182833121, 0.35457783956454314, 0.15087659393107355, -0.5250629631873936, -0.181889655327788, 0.6385405923897167, -0.47745412211899807, -0.4064845209362915, -0.6917231725524585, -1.5126272677814228, -0.4318287694532303, -0.6882691780402641, -0.17234890086830193, 0.2956604620361619, -0.8476149485389242, -2.134029759464182, -2.986243746495932, -3.288706998359296, -3.4958820381583475, -2.4756105161266175, -2.3118218550551286, -2.084820099726138],
[0.08917969318247616, -0.26321426515553165, 0.4007786597675641, 0.11825079122611204, 0.1890146154247517, -0.07551720613413558, 0.27381316375323755, 0.18161937585156948, -0.41499414687874875, 0.13979896042646298, -0.5457651260543397, 0.009137111915759515, -0.09165115004530029, 0.3857874922485103, -0.09068361135914209, -0.17655046038813907, 0.05860304459590846, -0.1922181700015041, 0.0224468272139417, -0.24424576779978358, 0.08076195239077429, -0.30618696752581703, -0.791609024951537, 0.3277660264119771, -0.5253776614556599, 0.21232010626049982, -0.7048694253801118, -0.20499969013108318, -0.42249042009961424, 0.42458939586662525, 0.010320308161923464, -0.7923883090784559, 0.15728932809747756, -0.011504812204430323, -0.17287782035004048, -1.3536675208329139, -0.10782105529869651, -0.12873991253999187, -2.015304858504847, -0.23967434770102528, -0.6203048372417113, -1.4031709010047788],
[-0.004102951122672002, -0.08267141660700976, -0.06166177224788122, -0.15943074759621712, -0.03538035401920923, 0.01958209424010969, 0.027964701972532983, -0.2072737018486234, -0.12248784813494516, -0.12708884903932363, -0.3047142796292587, -0.11234477102984855, 0.11684338784428724, 0.04656307116692879, -0.0525754590793734, -0.4536908095745574, 0.046633013448697636, -0.6201625306561398, -0.28933531905404414, -0.5068371132739194, -0.24569169049739908, -0.06003311691671903, 0.11919635447357466, -0.49957715342005204, -0.6186162713723049, -0.3129148000455468, -0.031867728991237514, 0.06953578119574232, -0.4391395940801058, -0.6020178195263058, -0.36458374888902595, -0.06219087728555052, -0.6312957482611259, -0.7123230560863135, -0.6191192111120427, -1.367403815169, -0.7068793782971452, -1.389529193381341, -2.3580287822903365, -1.5292347107714395, -0.31052280400361004, -1.5803936581255247],
[1.2519240856996263, -1.929744979584927, -0.04360085905810981, -0.7723326353509774, 5.2285286036308305, -4.3963449858248085, 8.753503666715632, 0.8185080417558936, 0.11061341520026091, 0.39529794777524807, -0.6693456633839876, 2.6208292281404044, -0.392563170880618, 4.978532015421452, 1.1661155908620198, -0.034944137008312816, 0.9387931457147976, -1.6308588517253577, 1.4976470810625768, 0.11895090128151849, 0.8817473867652619, -1.0646263194717374, -0.12306667358622282, -2.0329383391482923, -2.0478051845963554, -0.5685230265908605, 0.7600263109667952, 0.5662328077132663, 0.5512046636426796, -0.6174141644312955, -1.4616434778891372, 0.2294714638999378, -2.28016128601594, 0.913554179429105, -2.517994806985082, -1.1805230576049976, -1.1234616018921013, -1.2469118357009203, 0.836876814793948, -4.338229861188179, -0.9515019718877463, -2.6315900024700443],
[0.2255742122524791, -0.020077193887451182, -0.5414604890971371, -0.2544782440258654, 0.07325478781255626, -0.179225946700166, 0.004910035039568767, 0.0740968494423371, -0.7062118946663455, -0.3704531217746758, -0.5123208817074285, -0.22186093775968302, -0.9197852530449463, 0.21553200692122668, -0.10807070960352115, -0.880666237371807, 0.14615382179798586, -0.04640289399199121, -0.06114132412721534, 0.18037101601183617, -0.466916091235809, -1.2322458315869014, -2.4667806248914914, 0.8966965560049353, -1.059917937483361, -0.559702205699209, -1.2046164296430975, -0.5692003896934342, 0.13895780049229722, 1.6932040816447804, -0.6511487756803919, -0.7494958004154629, -0.8154670258152368, -0.3715596850824017, -0.8020162383859005, -2.4015667546202377, -1.9727300098759109, 0.9411371222748083, -3.2282949923508664, -1.2073682538799273, -1.1562545943775429, -2.126927511294435],
[-0.044796026053926956, 0.009924310609705639, -0.18551603736622066, -0.2696422956467974, 0.05555224605693256, -0.0002872042438605303, -0.12296658366316482, -0.0022513574501670337, 0.08250128252381149, -0.039750398524821914, -0.21479616613717223, -0.17270776340742666, -0.04338713122836821, -0.057818796468181104, 0.04689966216198445, -0.154722694753127, -0.0880726674917101, -0.2613017456008336, 0.059543590704253586, -0.11185331485596736, -0.24195911937353096, -0.19046747659808308, -0.14015694155578695, -0.17916920400793848, -0.37721772367917605, -0.23800654880882957, -0.12859987466930622, -0.08275778846024281, -0.6511186539746762, -0.5668895417130808, -0.4687618712462513, -0.046983455216154164, -0.4868305565567244, -0.47569765712350487, -0.6835546702664994, -1.451693093954526, -1.3720459233570517, -1.190170807760268, -2.332554153007719, -1.351550137421254, -1.1562353867855018, -1.5020330788286702],
[-0.03113309953297025, 0.09450013820856777, -0.1894047381339914, -0.025637408968862085, -0.14306766537297022, -0.029728690857090193, -0.05371660237689122, -0.20068794050969904, -0.03353367383183577, -0.33467239017794354, -0.22794564864725417, -0.03889489358738558, 0.09942027677611777, 0.1134903507586608, -0.1117494434090964, -0.1122662369960445, -0.022997775740790902, 0.0032811364354559656, -0.10266486216788098, -0.20237643636092423, -0.21791042745547876, -0.27534943166046333, -0.1763331893825538, -0.3143590010182206, -0.13304363820455678, -0.2470249285533783, -0.14621392588723328, -0.24184521145384683, -0.41768360290213563, -0.28412864977189917, -0.13229996733538096, -0.47351056693937016, -0.3791586842104884, -0.4631252078280587, -0.4804780093201728, -0.15265401925813832, -0.5618738707289646, -0.6321672284607233, -1.0461314662626053, -1.020498232593841, -0.749773983490553, -1.2770825762980356],
[0.09627926805018897, 0.04739879890561693, 0.03333066803048042, -0.19403801194821346, 0.06740807107962554, -0.01806174283518468, -0.02526427904616234, 0.046400111243546274, -0.046125518607974696, -0.21366227464264484, -0.0904074155907055, -0.2067738194481385, 0.1458143648956875, 0.021822987993485556, -0.24246163658018668, -0.20194031027980708, -0.23056631654839763, -0.26960485346534224, -0.21694585461414748, -0.3402604740333732, -0.23997179709821131, 0.032016361768448034, 0.10763490914326734, -0.09727669569794035, -0.055619790529815145, 0.05715866860331086, -0.07394122651491024, -0.16883163158716025, -0.058590675556791844, -0.3683702250912576, -0.35776487948097463, -0.25068527237568733, -0.19480757474221022, -0.37133257261112407, -0.28713273492850744, -0.37792793676123804, -0.5381923175147619, -0.515047600420168, -0.6538556334821274, -0.44446656612957686, -0.16138121059825128, -0.8736148830703038],
[4.43201204423836, 3.872760407389115, 1.3905125856833223, -4.50140220581348, 0.3262521408876601, 4.050298894008125, -0.7642365307277106, 1.459326761297915, 1.6203593353654695, 2.685688577182226, -2.36758397688553, -0.7897589304822472, 3.207846343478431, 0.13005614732880702, -0.22248997016847405, -0.5003091890026841, 0.6700152035293884, -2.939970639259565, -0.9463187066375908, 1.6977347484198833, 0.9162683334002002, 0.7259865692028644, 0.04205532558084049, -1.2901472500250968, -1.6964980209726088, -0.6578812362074846, 0.7146512382141669, 1.8814256155075477, -0.6472912656967169, -1.5715332235172252, 3.459030356467032, 0.9563211193278083, -1.4691239136344119, -0.7089152453781562, 0.005459684943730131, -2.9161360804299026, -3.6583927363161477, -1.70601001695616, -2.4234914905319425, -2.7084563870855556, -5.459265521512621, -2.2713683823035016],
[0.22835802058792545, -0.16481188312286954, 0.17288357885450353, -0.04733639178160357, 0.11182009247126375, 0.09168543364538749, -0.13151734017050223, 0.10258268252697758, -0.06194701581661848, 0.060795690864670356, -0.2507985353121904, 0.09717521515570707, -0.12329895319089489, -0.14925554613348818, -0.0648354486419391, -0.09981555016449485, 0.004593246169375249, -0.13537086157573017, -0.1406217305016427, -0.14662614550019182, -0.30987002473791203, -0.2892941268127957, -0.4182047737517006, -0.26076983035473383, -0.4214166290160867, -0.3135438117363587, -0.23053172697589377, -0.24080552733442853, -0.6496255904334597, -0.2759302423796438, -0.5019706250511845, -0.2665390890118757, -0.4156242809607145, -0.3956697117640668, -0.39860746522626084, -1.82838624603938, -1.406141604439724, -0.8755537500318732, -2.22272415938358, -1.0167760584623409, -1.0567376397141715, -0.8378480296667402],
[-0.25031978446729586, -0.1470788982936908, -0.04526612144567025, 0.2657443489856202, -0.0460123798716327, 0.3784515780053047, 0.0684053921138015, -0.22512739857898298, -0.34503179053535954, -0.3329770548065494, 0.03206441562529449, -0.009021692964670243, 0.1471724081360272, 0.08613124771594406, -0.4270200225790368, -0.029024261904359284, 0.07634074014697922, -0.03381512356392536, 0.0886046655600642, 0.07650788371658124, -0.008635075543788881, -0.2347449345873501, -0.2108539435929854, -0.19472157524263162, -0.13756540445211696, -0.26107377217011163, -0.2326821425964733, -0.1302205347112289, -0.17530375567372442, 0.10981520069261493, -0.13324507078459635, -0.39894790439658956, -0.286951518036446, -0.705438768859475, -0.7756675462155093, -0.2176702449475917, -0.8513327441010001, -0.5270729199274996, -1.364291166834891, -1.8250210465154142, -1.4174871605575508, -1.2981679820143253],
[0.04325725649330572, 0.15192565467980115, -0.047460622530954594, -0.22715653740366884, -0.5024954306550705, -0.033465632679977236, -0.04787937749488742, 0.23677687074920353, -0.02786848028526195, 0.19256623999329112, -0.08761555294919024, -0.43353013072644603, -0.17135789452948375, 0.40232165677194526, -0.013335375431500304, -0.27185491748448287, -0.11300802531096371, -0.24211456674628135, -0.2782731241002493, -0.303469152140997, -0.5875470080183944, -0.2283336291526762, -0.38067698909809194, -0.38680358827938294, -0.503338800723988, -0.1185612870539225, -0.5971344606435836, 0.09897236622940778, -0.5843910232542069, -0.7784442887573858, -0.6453667013262513, 0.1033571829852469, -0.19846843971902192, -0.8228979891461278, 0.5905610463772739, -1.877764337305312, -0.8446743862450933, -1.479100051214726, -1.8809569339756043, -0.9334476027443971, -1.1423197555785531, -0.3448916431869837],
[0.15221277684449608, 0.09172399076492711, -0.14453200452329856, -0.029919713181637327, 0.03572448291009997, -0.16182338414356034, 0.18596298861262558, -0.19564587094105163, -0.05691408074507676, -0.3771385451758428, -0.15256068672160522, -0.2535003842790266, -0.296390192560267, -0.1987045652763141, -0.33538199604875185, -0.41719425088026346, -0.08396211475352983, -0.38323725516526086, -0.10810095469850851, -0.24334855157803487, -0.17795350384007383, -0.2417643261211948, -0.36171335535005394, -0.3080639738890103, -0.44209000183166103, -0.5358906817313407, -0.368375189319856, -0.05858898313730844, -0.6992337058031409, -0.8052300356511566, -0.7307321720366833, -0.15441096725630463, -0.24987913852689383, -0.5359425639310083, -0.32177247294236666, -2.0849155128623083, -1.4243944949572802, -1.252888406180895, -1.8488964638827268, -1.5772228320134902, -1.2824175847036952, -1.388129892136491],
[0.2794805677807916, -0.06979672710543858, -0.001929333614686234, -0.14369223268328418, -0.36709089961099517, -0.16363930663421616, -0.054101084755542345, 0.0892254962774701, 0.06454972739544267, 0.11148039450178455, -0.1582355059912913, -0.19729651374110346, 0.09867390630508403, -0.31178915891188663, 0.06295690051042464, -0.17098627286569276, 0.0045545737201504195, -0.08055938163999439, -0.08395241247911289, -0.019924271697143205, -0.09453714454011257, -0.17696090727732008, -0.31311618569569866, -0.17469829807572845, -0.3047321021959367, -0.12398084233596667, -0.08626949832767054, -0.14303766489889339, -0.6300036734548713, -0.4464969489252495, -0.4485985339798992, 0.10708638113949731, -0.22032522680055153, -0.20472400985562034, -0.21762058894984754, -1.4154681048165119, -0.8811205936485127, -0.14963171911372006, -1.791281121106684, -0.36141995813115135, -0.43228444021109247, 0.03562340497161553],
[-0.06279842281289856, 0.13725622895057665, -0.09705038755202243, -0.35191119456554243, 0.09158563260394875, 0.13098195586121328, 0.21853374732576708, -0.14698421927238345, -0.1771279630239638, -0.07335904887923146, -0.6142642919864968, -0.24520976431442593, 0.07063671150610508, 0.04422864278708013, -0.05832833892216725, -0.22046240601768213, -0.01631414332175508, -0.7676176682339885, -0.25119575805979455, -0.14260621578617041, -0.19957661288995296, -0.39232215579344343, -0.4470317708161652, 0.059020264372071495, -0.6133907635084273, -0.034339014731794494, -0.2386402330286388, -0.4377625365604573, -0.4551875970490879, -0.9294657615767824, -0.25215887240955437, -0.057056267703146486, -0.36949090718511185, -0.7758586753011096, -0.5945590191148317, -1.3541244730662303, -1.364199903014491, -1.1861027370918118, -2.02920852697027, -1.3940373283296505, -1.1715489005612578, -1.5596968546528294],
[-0.4708866002006292, -0.260741068769535, 0.6006272600472902, 0.6652555465956622, 0.4431987124747919, -0.08483105958518483, 0.006257357957053432, -0.20722261984067147, -0.16243163136028416, 0.35061014495358095, 0.20674585252930489, -0.005848103334025453, -0.029830141717500737, 0.11503956822410771, -0.40565720060022925, -0.48997773204026107, 0.195609967570652, 0.18407728306709972, -0.05619034860011251, -0.09505016534240628, -0.17756999930299197, -0.14456603592713746, -0.3013281932805653, -0.04715495319255363, -0.32224389700735934, -0.10260225122833376, -0.49557707903294873, -0.19119085587301568, -0.730360337851585, -0.265197840870072, -0.4230209611611019, -0.41010137096787447, -0.5956853681116004, -0.14782267149156292, -0.7873529804224514, -2.567137070061364, -2.275976851418611, -2.591679933856033, -2.8014022601124755, -2.741653286578656, -2.469801449370795, -2.6382959211156254],
[-0.04484095281408317, 0.13239572976100353, -0.04653609841042081, -0.13867868993301705, -0.08247779185760734, -0.08510005512920657, 0.09485958254345052, 0.007008865663119327, -0.22433721277651084, -0.053607825221262194, -0.1385546123453659, 0.16231050980899492, -0.19367604945139544, -0.18850200441623924, -0.08065718989994793, -0.29781317288472675, -0.42248720754247876, -0.28359333379556295, -0.15908783235763777, 0.05981617834733024, -0.30027376355452046, -0.4192446701094187, -0.40709910352106304, -0.1913545650738321, -0.7405609169678377, -0.2594098600392031, -0.4473953352554768, -0.3563730152215907, -0.6683199619738592, -0.22968134982512228, -0.8922377797374831, -0.2832006614298049, -0.3713747770032319, -0.08002414574628576, -0.49185919843772824, -1.525590702000648, -0.934244407774509, -1.0401571519300923, -1.6011884015060833, -1.1020826087020197, -0.8950785664834331, -1.092476592935047],
[-0.12229337978943118, 0.4583161876344496, 0.5800341970052828, -0.13599680848080795, 0.49747340380401717, -0.540097528927475, 0.03870921638736499, -0.05774030464957453, 0.04351595205814208, 0.13904155794328962, -0.024436617344585465, 0.09404122616478373, -0.4165655099213396, -0.12388001890091475, 0.16945065867728226, -0.05676195065081224, 0.18695498619682568, -0.5690443695905398, 0.046535304269124035, -0.022283781967596235, -0.14531074110177733, 0.06414014918655472, -0.16662804088010003, -0.48214207637304796, -0.9321298167437955, 0.23055762122375736, -0.6611316744262886, -0.3040546231900157, -0.8972137054251715, -0.4287579455850442, -0.8121530199638375, -0.3344966363444477, -0.2706925550865991, -0.37159245234876653, -0.21324012420650923, -1.54119284954806, -1.1415138018809432, -2.0539329333404424, -2.3583060309435004, -1.373754464031945, -1.3570999014242848, -1.5246787165855804],
[0.2291283830806966, -0.27431127281643336, -0.38090452756647575, -0.015017158532495582, 0.32672454876025936, 0.1657829605098255, 0.25987299589980517, 0.1762074196759346, -0.31159070706265357, -0.5940279109199498, -0.34324547622480633, 0.33046992758057764, 0.031051675280967105, -0.13811194895018497, -0.07486743691708628, -0.18791201699095816, -0.26105281522558, -0.056084195182660404, 0.15240340678254044, -0.2344351014060621, 0.03502761695491935, -0.023087407578308967, -0.2900176753947214, -0.5668546397310191, -0.30384521044481905, -0.24608701199209487, -0.27994375528998966, -0.3249033594481029, -0.25197623622338644, -0.8562653112156217, -0.3210673679910855, -0.3705159381363003, -0.672606326363942, -0.9461881998206454, -0.7238664892359571, -2.392762701721054, -1.8182440806168858, -2.0232373330932196, -2.122547150427178, -2.193118586787918, -1.90245896689268, -2.4517753859403117],
[-0.0015206403251002637, -0.37437595170758703, -0.6320300801923696, 0.06841915196506343, -0.1050099972992359, 0.41542963935758376, 0.307744615015451, 0.5440594564573619, -0.29199217697085667, -0.6258049333573349, 0.1687885478144928, -0.06884740264778695, 0.11282559551263291, 0.40067835036332156, -0.29551356360956504, -0.4443553261771868, -0.2606807680490878, 0.6058568172397806, -0.346870577411952, -0.13529396942311156, -0.016655693907329874, -0.5723746686873009, -1.28514214697042, -0.09342753404790553, -0.36751056681707933, -0.48287410165004774, -0.37895774691488926, -0.23124230503426357, 0.03928105643768641, 0.09396968085770901, 0.8019052754553939, -0.190068124775844, -0.37283507617213646, -0.7716122588388306, -0.7127588545811485, -1.0231874181685592, -0.9121951914121981, -0.21435925960149016, -0.9055941962354647, -1.5807593373616535, -1.4019486461227828, -1.6033345448991816],
[-0.2060145868558603, 0.2980061843404959, 0.1156945102372975, -0.06149811701604343, -0.09596661791985998, -0.23856395179460194, 0.005151933325848117, -0.11620098306110582, 0.09287962020782957, -0.01567336838000339, -0.05877116060000168, -0.21857856054979122, -0.038969784743368395, 0.17757796449433214, 0.04385842001625927, -0.18977979585546464, -0.4230894964052294, -0.3835408284678424, -0.25398179879017047, -0.08322090527146447, -0.12131798676068892, -0.2691475659493673, 0.14193642595439881, 0.09460266359266754, -0.5886217158226245, -0.2927028109297755, -0.557050000631455, 0.022689436234590164, -0.619894263939947, -0.10362879513645314, -0.2817529216300446, 0.14136918431875822, -0.4297473012197432, -0.2316837485243169, 0.20615449804606226, -0.753442584640194, 0.24597510553192908, -0.080253848937049, -0.7195301108502058, -0.0903952892068489, -0.8054140687101347, -0.5867758151062857],
[0.15902242406021955, -0.20355937070870345, 0.2509872184525435, 0.45969574114692247, 0.2927085924835664, 0.09081686432746296, 0.19019798348603417, 0.18307432891509928, -0.06387542185955127, 0.07522726463056316, 0.4938258788293778, 0.08907739314136046, 0.36663308894769364, 0.4238918052898606, 0.34122114829259703, 0.1652977324738714, 1.0213666284432876, 1.0323285416134016, 0.11350565682533045, 0.2776588646829625, 0.46565377605903374, -0.11853613086951645, -0.807813038020975, 1.0333072774252914, 0.7340478745933652, 0.33075653962597734, 0.33591008561211894, 0.30771451012120066, 0.784969735660103, 1.1270691538869302, 1.2344548273915614, 0.09094545579490988, -0.1430751373263449, -0.023968075407759918, -0.13843900808641255, -0.15932986075503883, 0.0906338447055092, 1.4427647823730503, -0.9504276458059855, -0.9767803181940892, -0.176517354520675, -0.27787685655966715],
[-0.044535114480093986, -0.12225128632993841, 0.06088937067063211, -0.2114451253337925, 0.1405712363273972, -0.06364683901663605, -0.25038578702089587, -0.1946343158023113, -0.3219053898400121, -0.05702858008266488, -0.23167281464211842, 0.02300533569323265, 0.2529517485519827, -0.4566959329543134, 0.13181157943455285, -0.4730620756194149, 0.44108708090395765, -0.03759405033460269, 0.0703092892479691, -0.3232590512365715, 0.6769127637755629, -0.17275275524639852, -0.5512947192376033, 0.02557434876553949, -0.27110281226315364, -0.6637499466667217, -0.2073892809807123, 0.0007881952886723994, -0.55323401833617, -0.6271079302718797, -0.11433275992332978, -0.20160061399887647, -0.9260040843417108, -0.5046322919765908, -0.5726538126120064, -1.3159857562225001, -1.1807489547827281, -1.5756259765671456, -1.669180206036353, -1.176900887273488, -1.5771791239559363, -1.363609273983212],
[-0.4453239874328778, 0.07705949650179407, 0.2559823544880304, 0.10102767996290365, -0.09889106813004114, -0.021762309428801436, 0.29080288103637686, -0.3856737765909369, 0.2819486262470345, -0.1524700491227143, -0.03470288804687241, -0.040870111737568986, -0.06833670090437644, 0.2852063908900234, -0.2537181169900987, 0.19531210129617121, -0.16436844615392662, -0.22064238639918235, -0.30625678835017894, -0.3304106533622743, -0.05682428930091383, -0.3472791525187002, 0.0050607862310876476, -0.2445732577007172, -0.32668844441178635, -0.21549962821403218, -0.585915912693358, -0.5235715291272139, -0.4279666003807949, -0.34367999724153964, -0.2314295087043121, -0.5201189727822978, -0.10221199733623483, -0.6246233785814024, -0.7821589305737986, -1.8845065661526275, -1.6472470288282672, -2.2348079278076387, -1.8181490248444148, -2.0622541673245056, -1.932381018042572, -2.2614905626893673],
[0.04065753705058766, -0.0015393854162503672, 0.08339077506542365, -0.017583054285323193, -0.07542923521290575, -0.2452700044605009, 0.028076600868154198, -0.01684443433558638, -0.09803847411467503, 0.055411260757076156, -0.028808643265423522, -0.2645610119384447, -0.026259694605770916, -0.11030310189945917, -0.1773198482826913, -0.1347497849532053, 0.007302531602776921, -0.2026971560311683, -0.20456665321852946, -0.25218326288091364, -0.14588761090541016, -0.1494445992853012, -0.16200008549268594, -0.10874572900405344, -0.44071221666665783, -0.31080043023836323, -0.4348764029837848, -0.12447811581895048, -0.7295220232473988, -0.4241979545305413, -0.5168918789461008, -0.32589131929216447, -0.3708650312287217, -0.4050519449594287, -0.5930988004394749, -2.0491680091983095, -1.7730476330284748, -2.0723033191398, -2.4659592425712056, -1.9380079003469897, -1.8585271878067773, -1.973185017489535],
[-0.12421804373603298, -0.42694517766501483, 0.09657588319707659, 0.20681041801233488, 0.015999466242897543, 0.0038118263912345117, 0.05071175434263809, -0.08588301190089329, -0.33038642644560584, 0.01494900036297929, 0.07079820771501846, -0.02868412283940694, 0.018644963248051257, -0.11634558701510665, -0.06385385187218574, -0.15807831687690074, 0.035061671591661715, 0.010765529483389552, 0.08867564717787264, -0.2717658256204067, -0.1880577592830178, -0.17363195451767996, -0.3064368399118479, -0.14027929171896927, -0.27128963079166585, -0.17016309910357338, -0.17836126535555047, -0.2517144640106817, -0.5522330062386233, -0.07540304703831736, -0.4475285041110648, -0.539391155005295, -0.47573468757124676, -0.4271373563171222, -0.758800872925746, -1.229373992295898, -1.1689746738750713, -1.1998198065637316, -1.936284316080164, -1.6569333439738267, -1.4809371737080068, -1.4442012810136808],
[-0.08104424402343917, -0.22422001066297986, -0.0004284300984471623, -0.23269102085188612, 0.09472185921121859, -0.16089433781279985, 0.02798928015329115, -0.017008644730366115, -0.2701168913950732, -0.2264800890730066, -0.223226740307587, -0.0417787495786379, -0.472910125570414, -0.2529167798424164, -0.24373099754954208, -0.2400546603490825, -0.4872670896357072, -0.8127593763254822, -0.05504565096880614, -0.7648803183850876, -0.273146257572683, -0.1310922423812387, -0.2693030934063554, -0.4475926921268047, -0.46007553288867925, -0.2993038180991082, -0.7092566342438634, -0.33735210903976987, -0.44437027670768225, -0.8564356308570014, -0.8124517232219145, 0.03756106279549303, -0.6028828905832451, -0.8309919492738699, -0.6671688180597155, -1.5924122702135477, -1.1804480628250387, -1.1277406664905918, -2.0027393412644314, -1.013682871690066, -1.1678379433924855, -1.542638677183466],
[0.0619340127217027, 0.21335610521204137, -0.2650501858269215, 0.5752464619582432, 0.27704183701649676, 0.3590327167907916, -0.17337123624373602, 0.10910683591711857, -0.032605985815612554, -0.5532102219210878, 0.44359652214823214, 0.003895389381684751, -0.08993441268938039, -0.23948131649531337, -0.19843180748418116, 0.16766518076552892, -0.5555900239381099, 0.3148562857292429, -0.15833656850279262, -0.14699425381582695, -0.030909557469972895, -0.24213100105332372, -0.3569151618222619, -0.49009933910393166, 0.07389653487487678, -0.45684479892069274, -0.03477980304115765, -0.2791833308112009, -0.77006938659484, -0.5272890377664234, -0.7172855160471858, -0.7379130987799881, -0.574638092481084, 0.1954537168034736, -0.7105319493955389, -2.3661719338075047, -2.1284754893252265, -2.5702053055975966, -2.6190737964984754, -2.586097520548482, -2.133492387890236, -2.2268167380941972],
[0.03307758171426434, -0.055687025143733575, 0.222882468664032, -0.11366866320036924, 0.14117901093391974, -0.14643757999937287, -0.05374539652724058, -0.2643501406572116, -0.10966126977988773, -0.004089378584574096, -0.1553307960536335, 0.029976019903584884, -0.20477453544366322, -0.0906887632493771, -0.2467796929995653, -0.4531610929507082, -0.10022124047359277, -0.23324806077832713, -0.11464107083293634, -0.36248503695678064, -0.24994224153498354, -0.14866820593909075, -0.23213085547247528, -0.15146063952539968, -0.46783090862374754, -0.14293868147586866, -0.1753314165636496, -0.07609952268413185, -0.4926974474640503, -0.7182048123183555, -0.5742524655250657, -0.2555200334903114, -0.46762647887334413, -0.7299867066241962, -0.46591862673802165, -1.4872572978339427, -1.3484104701583879, -1.4290644912906953, -1.8761387511212466, -1.5633171228115015, -1.3746088515488377, -1.4629083706228088],
[0.15152512699334017, -0.11196763668392, -0.1702778828217774, 0.27803621012726865, -0.19469097760252957, 0.4828059731649251, -0.34148555590231267, 0.059266051035630554, -0.17666100917705, 0.0652988898641973, -0.19522237511722204, 0.16615448608704603, 0.434419210114441, -0.32585517472265346, -0.29826356245541424, -0.08899943265460179, -0.1623752972130764, -0.075172587471546, -0.2980577383463396, -0.05490414112528661, -0.19037413836001676, -0.2547612084097222, -0.3220579885855687, -0.06483111503459209, -0.13863455588910822, -0.5037385796279801, -0.09454873542718839, 0.07729216594268189, -0.3345804341070537, -0.3816630530230242, -0.14297273515202763, -0.49797430563649875, -0.5882222671960797, -0.7056406449477973, -0.7721566442668784, -1.3737815649610403, -1.276303205004625, -1.4305147626613255, -1.356174795439517, -1.6939674557687903, -1.6538630051447922, -1.057484138535167],
[-0.08989949566808436, -0.14175573296685895, -0.40009552338572535, -0.0327867039548957, 0.32246857116170696, -0.15717246996420664, -0.17060102728007784, -0.13469921267203658, -0.373411737096145, -0.3721758700660867, -0.3111642690938264, 0.20602246994282433, -0.38880717552716276, -0.6259918470404913, -0.29001703174601073, -0.5760356716926065, -0.3112334327012653, -0.33219185710123167, -0.1011140798720295, -0.38568951056360307, -0.22270844962160577, -0.3742463218906451, -0.36472215067717006, -0.2174638132677086, -0.4766576013023753, 0.5246275874883052, -0.6141685758888151, -0.05595536101910131, -0.5097425632213991, -0.7779983956814038, -0.20571325969165133, -0.585215337926522, 0.09023699465228889, -0.44193548093678936, 0.11409879540842763, -1.1984366061572973, -0.8878131034826394, -0.87947833604108, -1.4089168921591972, -0.6852371648855353, -1.0648875300161291, -0.534128893098299],
[0.27541353824564335, -0.15240081111380263, 0.13527669854243365, 0.27911294175558105, 0.0560633600511709, 0.011909525047524041, 0.2950515185598472, 0.20271551761122714, 0.006035244873290496, -0.15383214487082372, -0.13952229498602578, 0.14882466852176118, -0.17398186484426453, 0.07555980894611367, 0.16954536308423782, -0.0010089678372416206, -0.4091375174053615, -0.4370874527345977, 0.12689985162563225, -0.10392834249835697, -0.17038976726481134, -0.2701786206674827, -0.2178467139200442, -0.2560521518838065, -0.514620288243075, -0.16891607257695296, -0.5324588603895489, -0.4396478834713344, -0.5690330255707842, -0.25378044951386364, -0.4738940076562994, -0.422707585300076, -0.3155911914122251, -0.41007336815191886, -0.8380187078935247, -2.093750271075291, -1.5732874872345308, -1.6635913777850575, -1.9761213143714225, -1.7330375001596856, -1.4457321824964449, -1.844771803229224],
[-0.025281247840876323, 0.28541539086274514, 0.21571262849672654, -0.23299897717858578, -0.15735246154995525, 0.11377384405131315, -0.09226628326127843, 0.06120090341853266, -0.09074265096834055, -0.0011221486795657736, -0.3134910920062955, -0.02503275243646967, -0.07287270826031332, 0.015749958884608495, -0.16869394088160658, -0.37969202163355525, 0.25598848896113446, -0.3744556286923753, -0.3152231112132089, -0.2190266411431895, -0.2164619300206101, -0.3583044490573632, -0.31373049603596104, 0.025695000983311043, -0.44443014915701, -0.19780758128202536, -0.3427226175334938, -0.21944248603973077, -0.7981797158368386, -0.8708536067026651, -0.39255046299617785, -0.282930778353639, -0.5809013244655892, -0.4556469525249787, -0.4770656894838596, -2.0035786519177172, -1.5713336878136899, -1.808994285822909, -2.1649029702597917, -1.5722809444898174, -1.6433147503220988, -1.8117330353457508],
[0.18542397069677138, 0.18091433239621799, -0.28282054519630073, 0.07071159480405936, -0.08637689498215262, 0.14201229725549172, -0.0312965367466732, 0.1380115437345596, 0.09887946458582851, -0.43584238468324016, -0.19976509339590912, -0.2623837139912022, 0.09916380112535364, 0.16769274981052476, -0.34218988469142153, -0.3156070650423583, -0.6678620673898777, 0.07578246544371106, -0.061331240908784446, -0.026008272229236712, -0.2143169393214645, 0.057581148049886476, -0.22148883886788065, -0.7812380844695954, -0.00852429953603385, -0.28511412917573126, -0.08732768148662941, 0.010858811662225151, 0.0044267959333404465, -0.8874443941439114, 0.02530501535924944, 0.3389766148836113, -0.41362044586227636, -0.2587126522798882, -0.22303111371483866, -0.5947178982873078, 0.607726119613658, 0.12079185731957728, -0.2706239488528071, -0.7207481221048069, -0.26883521089630297, -0.768176380431922],
[0.0979545844791109, 0.04521885520447128, 0.22514125203995972, 0.28699642312270207, -0.43005068688844933, 0.03851074075968362, -0.06811889513653772, 0.038032344465820184, -0.35112578462778304, 0.02858328122654128, -0.012412465099384217, -0.37241563599841127, -0.21424602307662738, 0.04546533669928419, -0.17053341191711, -0.4598007167920965, -0.38637639885948055, -0.2988658331591865, -0.570930604820554, -0.06673991874167166, -0.5033701355481416, -0.41050374453238203, -0.4997459688188688, -0.3098007258481459, -0.323284957005129, -0.926163668706666, -0.637343729785525, -0.04966810837771013, -0.7330974273306133, -0.6832244493090468, -0.8299847500441445, 0.13755533902652456, -0.2536941835311381, -0.46931141773525165, -0.6240551630687925, -2.3855366738465738, -1.8410263384219514, -1.4593721186604398, -2.32032757183691, -1.6432392699962246, -1.6432646896990513, -1.429705462179859],
[-0.005103102384573893, 0.5700367446260858, -0.40750564800630756, -0.013712085843816503, 0.11168197618211619, 0.2250002265714254, -0.6243456690524749, 0.0011688191551525507, 0.14393083801954146, -0.2834949645224614, 0.11703521870839348, -0.1081904726791435, 0.2592459561582379, -0.5300623593314356, -0.02724250147352231, -0.41917210902576507, -0.17827172199995595, -0.11285759384271045, -0.6433188533020716, 0.19238300431814312, -0.03125908916624345, -0.079289024372477, -0.4185804416011369, -0.2644900404413494, -0.11103046979659312, -0.6610744873622608, -0.03436258997059978, 0.1119027595737856, -0.6437067023652467, -0.9222498440709541, -0.38015070497293474, -0.3337202688992591, -0.505138629444197, -0.4303526558132694, -0.801106731828042, -1.553526359489598, -1.5870260895200714, -1.6604331357461506, -1.670627181523233, -1.2379415228009591, -1.56421695492957, -0.5515132635305975],
[-0.0641283001487898, -0.04379939233210839, 0.26962180351650417, -0.12453581810518675, 0.12140217682942621, -0.022733402873785016, -0.03970045459949424, 0.1592457124544846, -0.12377425020624072, -0.06816192139448327, -0.008769174202015072, 0.0014580895978170763, -0.13461409559437806, -0.019715421669342516, -0.060700427301258485, -0.16773434895514533, 0.04540450094482436, -0.05625363860609316, -0.20093154379264258, -0.27864819609681063, -0.22493628392785156, -0.27410058153559386, -0.212031009367969, -0.25337822892769807, -0.262787090164654, -0.29051939392379256, -0.2302544177826726, -0.22062136033173693, -0.7569236384366824, -0.37959018913759646, -0.5006353560001162, -0.311371811266719, -0.5721931159847611, -0.4841699157868688, -0.7089695008438082, -2.459978097366283, -2.162808047756063, -2.275023748797365, -2.7392381008498328, -2.4744254644452846, -2.197624083281781, -2.299955340692202],
[-0.5419468233813419, 0.03560024753930388, 0.36280406125105724, -0.19022771880330133, 0.06029321550935391, 1.6492215760738216, -0.13759661942946638, -0.48948067463394307, 0.059349398026310646, -0.111095844555362, -0.09551591168173038, 0.2672637787852374, 0.6250906551673986, -0.36290851361051, -0.2947903668503934, 0.3343040293776232, 0.07881804315680786, -0.1140858548768551, -0.10112742054941543, -0.5950797120985303, 0.0726514759611617, -0.15025985508245362, -0.22644858080760757, -0.28666913429603386, 0.19023417822214678, 0.008144091443216905, -0.8617696512631711, -0.44257083512199696, -0.8478294166382373, -0.6686292456950528, -0.22700813915844664, 0.2310126087117033, -0.701386476707745, -1.2008662162464103, -0.8797080064836854, -4.081018075780308, -3.77826472538445, -4.126673694039032, -4.101939183552342, -4.221215416342805, -3.990851495149376, -4.225425623102951],
[0.05163891412228341, 0.020362004809444237, -0.14121695773113505, 0.16674750290154722, -0.23288359638978479, -0.03512818596041573, -0.11080809390299819, 0.08227193173996931, -0.1907920148771567, -0.1231610411429964, -0.31706427387929353, -0.37931846485432485, -0.3786482097517424, -0.044092707310771445, -0.07832414802208305, -0.24282147040686958, -0.5039854895388808, -0.09108530575001376, -0.6777014267886854, -0.20822946267624382, -0.33924864185042014, -0.44900872129321545, -0.8704007554739419, -0.09364419165992766, -0.44142703555901147, -0.9037469605211079, -0.7351706319653215, -0.24616275802342022, -0.6939654411764054, -0.4515919044517871, -0.4558704778588767, -0.05046158215321646, -0.5684158402791137, -1.0552326303229778, -0.6830362040016132, -2.4957801724795114, -2.112332737723846, -2.201232086445909, -2.8789441327414176, -2.4118177147462654, -2.0063455201577187, -2.4177055128268883],
[-0.05244206144988943, 0.3639237762016029, -0.5612174467554195, 0.0680705336958913, 0.498313185590247, 0.07051929772714244, -0.35218297869879645, -0.28513586591202594, -0.002272601560086151, -0.6546399888487789, -0.09606111469404936, 0.23743710177216346, -0.009354259243331962, -0.29744178717211667, -0.2904133840294038, -0.19998547305274655, -0.14246126264088607, -0.43099640413687856, 0.36709998823593415, -0.23192427738103705, -0.34590998346981716, 0.2788541506940645, -0.20639410950452375, -0.6627150321682735, 0.10557366587717211, 1.2007571154911068, -0.26237994921205515, -0.10656832878655956, 0.47506879092534254, -0.8573478716707229, 0.17097362964889853, -0.046590889106888134, 1.4289563166591734, -0.706336365891982, 0.16327429750775235, -0.9577227143118039, -0.5886142582128943, -1.0913981190662625, -0.4714441146321721, 0.11712885523511339, -0.9356075710776492, -0.19171237878007763],
[-0.06347276684054762, 0.08949165223597044, -0.2275751146173781, -0.20146323361530416, -0.12634298142348127, -0.26037440147941554, -0.16988123263133842, -0.05600111680579049, 0.13203684045180236, -0.20505413872548098, -0.4203579091518801, -0.046174538561395514, -0.26676567501369414, 0.1831779214615405, 0.06501328050631112, -0.4724883278740819, -0.6718172963882534, -0.8206727588030226, -0.6058036601451929, -0.28294148212824616, -0.45825833833387847, 0.32778412738706564, -0.25983444831843455, -1.1022148995545913, -1.2514769334571498, 1.0532512138477086, -0.9617424246767391, -0.09501779252595802, 0.5418719247665403, -0.7505874896372406, -1.0892954354660425, -0.715296480932692, 0.7414922639373336, -1.3475741288919578, 0.7273329817387261, -1.4383639167073865, 1.340885006230189, -1.478005224551601, -1.58286159164998, -0.18951491855012767, -0.7470980716799316, -0.9522316795271472],
[1.0059076511319467, 2.771098246577913, 1.347654700665691, 3.1548601216227756, 0.1029257534884707, -0.07589647651915167, -2.136325920452993, 0.130995267083375, 0.6855904041136095, 1.0407637351611547, 0.8593925608909899, -0.16389456084937715, -0.5079270169864702, -1.578577324296544, 0.012493142308455938, 0.6949216155377687, 0.32364873884612916, -0.19937682105937887, -0.0005716119829003155, -1.3117658594055102, -0.9431891955103526, 0.22019332295256824, -0.2211144604793013, -0.9361727052257632, -0.46844788924226133, -0.20910784664685722, 0.30136177227099925, 0.6273415856302392, -0.8912384074814245, -1.2401699765960583, -0.9451721694610834, -1.101569279144869, -0.4568641523721895, 0.7637141074546165, -0.4689634970560205, -2.795523613674721, -1.8578087725756103, -3.6019263937897765, -3.2486265733990507, -2.7669168361667227, -1.4814894703067867, -2.113464328411505],
[0.29842453028914223, 0.6108800063718448, -1.0095464936348866, 0.22678583522770443, 0.42518097735118143, 0.44166755893722975, 0.15270016112122964, -0.024092607481980968, 0.28524508044626384, -1.1691172783595747, 0.3698522660455645, -0.1435375213011618, -0.01080445102172722, 0.03269507191460693, -0.12305149447509009, 0.0763110171237426, -0.8370682604987928, 0.003983681369032407, -0.07325386409411333, 0.15606457648207486, 0.13774687157714127, -0.3436309581904915, -0.4876392341858417, -0.44168789679457277, -0.13899842715711322, -0.23783837093455798, -0.5684505216186676, 0.003156863891863693, -0.49164272480952204, -0.9937471701303584, 0.19056499440045296, -0.6524729644602179, -0.6336654197175263, -0.39140249739718963, -0.5784965050370562, -0.18516462923598387, -0.9657937399496509, -0.9363287515378568, -0.7903459918782285, -2.0668472239820743, -0.7703165138186494, -1.4956280210577695],
[-0.25084501883733656, -0.13428299514355024, 1.3798172865875988, 0.8053166176185145, -0.6393426168058947, 1.346693367528009, 0.16989201997334444, -0.044075956333161616, 0.37534021308717824, 0.681284778495868, -1.130502148218105, -0.8517658321411072, 1.0814346606384515, -0.04947628202256727, 0.21926047689216238, 0.7388265610690875, 1.0658979669016415, -1.4631342640775797, -1.0399275663902467, 0.04806656134118867, -0.9437177982412795, -0.7953148460299001, -1.1848349349156975, 0.353619290304854, -2.224912391909955, -0.5331485007597421, 0.10862458479764714, -1.4740239585696011, -0.7576589251238528, -0.43066262467578936, -0.4249324532595319, -2.373781887468775, 0.5237368132917858, -0.5934405144690094, -1.111848351830226, -2.2965229298636105, -1.6378968015685638, -1.0265749712562695, -3.3759084864480804, -1.9430662714386713, -1.4144717515978988, -1.6790127438676796],
[-0.24275214861780375, 0.1775265659029187, -1.7364240665902204, -0.8692961071118497, 0.5574911952144335, 0.721121731304543, 0.6160901667827305, 0.41212201284238154, 1.2242928866577714, -2.7490098825565004, -2.628437230248884, -1.3047972043085307, 1.3485323198707884, 0.49807337982199196, -1.3560538673339901, -0.16240783370776687, -2.9116443951750273, -0.8478993187570513, -1.1846481944734668, 0.6972498237229379, -0.11395828710897206, 0.5266594868147205, -0.30944756198236845, -3.926092075408135, -0.19501337421226037, -1.2061331179811672, -0.021401052329860004, -0.1583064053737175, -2.7159666693907165, -1.559545449589167, -0.6529893752518923, -0.39700618315570735, 4.88970724361615, 0.2663338659304826, -0.8926800380790059, -2.8515407545385307, -1.3415272750646632, 3.3976142208727333, 0.42491268519012926, -1.5250283959183084, -3.8701125605516125, -3.857496263378082],
[-0.11789734171859069, 0.07612352425453361, -0.36374261905907523, -0.19622944702509115, -0.14602604895735546, 0.17986707051849551, -0.004952404978778958, -0.27777943765409807, -0.18298429095601565, -0.3133197145252584, -0.11790129684579734, -0.14793501213419427, 0.16692144970474534, -0.22709329445048426, -0.10959966596723046, -0.16306519171634185, -0.29710516480068044, -0.2529528337171896, -0.11580773555128919, -0.10595856092323021, -0.4605177461804014, -0.32336326611650423, -0.22563743304811962, -0.4878711656927413, -0.4259375003851362, -0.37380369395362567, 0.015914801517830354, -0.31405300011913234, -0.502578703773376, -0.49591531122839755, -0.5116652987940045, -0.29907539216102474, -0.38140133907390844, -0.06779241748309156, -0.8913253534744595, -1.5775123366751802, -1.2609995459767323, -1.231262447885744, -1.5545080698369302, -1.548144906045741, -0.9329264894378333, -0.8937619474026581],
[-0.18896025500691876, 0.9637113158817134, -0.5770063608154511, -0.7132416454457696, 0.3851042189711807, 0.904136140700892, 1.0339514413951396, 0.4536901059919612, 0.4655192078298316, -0.7411059135749106, -0.5904562321039898, 0.2243031372297898, 1.0899354432545227, 0.9719793502086839, 0.2135915971948218, 0.08860381710814635, -0.1359527681954147, -2.3521061676690476, 1.1632559161875051, 0.5884110798633934, 0.8269961088134808, 0.6127310585711135, 1.0423358005169157, -0.7883707471852657, 0.63609588053237, -0.3003438169727123, -0.33087665814783845, 0.7628165119259416, -0.040483886350331316, -0.7239258383829557, -0.6948615536261057, 0.8146721230624737, -1.0975219835267127, -1.039207613398292, -0.401039233978144, -1.7801184982758393, -2.2967279247792254, -2.2414615162082834, -0.04142212492544008, -2.8278150481502693, -2.1053363972432937, -2.6947061465684143],
[-0.09534315019538676, 0.0616032432898827, 0.09679115954524474, 0.10874366714210183, -0.09303003047021413, 0.28921361982161375, -0.057876347553798474, -0.11220448186033806, -0.10319646572696566, -0.045258027290837465, 0.018307353210870698, -0.30935120681413963, 0.20225962312439968, -0.23888607724804206, -0.19420143762253406, -0.05691830694738315, -0.1503240413888319, 0.009116761037880738, -0.13169789197883838, -0.09290361862495605, -0.32202794147831293, -0.2405107478738603, -0.13341688664321588, -0.13569480730878944, -0.18896021800968588, -0.5949854076849108, -0.21034524095519794, -0.13666882031888697, -0.5891517951357929, -0.29910608505621744, -0.661391283256528, -0.0455009086072051, -0.17080956505066736, -0.5482206608127218, -0.5091709699443364, -1.0049415862119235, -1.0819464927688567, -0.7236486842779654, -2.024352073213513, -1.467438989152982, -1.2432865596408043, -0.14558502290415087],
[0.14564244735316864, -0.06391263584739748, 0.7466342592509632, -0.23436795806892854, -0.4905780331083747, 0.12752199143230897, 0.44799303023567855, -0.12208291253230645, -0.25517606072581406, 0.21435109359683682, 0.2684870210494337, -0.6406398151378117, 0.35233006726063915, 0.04970519776734124, -0.2808167876724065, -0.20891018876624035, -0.31002661919245805, -0.4504982686309066, -1.6682582226637286, 0.027622508220040396, -0.4050635353687753, -0.381777175161506, -0.8320413476993671, -0.41119753972873485, -0.7321733512441958, -1.8126923678387177, -0.3277223273262518, 0.25654237312987105, -0.4862611841113248, -1.3394336783799583, -1.128856808808956, -0.03757706264048641, -0.3831190417627904, -1.3216834636246202, -0.6161775102875318, -2.5362682908793723, -1.5220793772237968, -2.0008954737220304, -2.107976199862555, -1.1795146390622269, -2.106296100266439, -1.6373402042155238],
[0.11044146170065487, -0.1513466208704818, -0.13622503542530665, -0.20568903850021844, -0.045277130773524306, -0.056906699999383054, 0.06622551113306983, -0.05069694544620324, -0.09226602463997059, -0.1593309088721122, 0.040906895925539984, -0.31964247427149006, -0.19651715467135067, 0.1035895033907576, -0.08919532417812265, -0.31329555094149436, -0.048850612411980805, -0.2638693005695124, -0.40530861150662467, -0.17101056894899175, -0.15166979452315604, -0.33888130722213144, -0.36102822114291383, -0.26169991826796474, -0.4393875046051667, -0.5237256530562865, -0.51753488185081, -0.05060186168380284, -0.7003655750419349, -0.49203226250811966, -0.6192689565265815, -0.3971700419359921, -0.30372238072102614, -0.31487371795218366, -0.43148365411111217, -1.954782450305432, -1.4023780743088703, -1.5326622829525598, -1.7181615210339314, -1.5508356820574256, -1.338247175214445, -1.6904370727984168],
[0.2356163654750906, -0.10677094730610341, -0.1605402858485463, 0.054411196747682744, -0.07360681350353222, -0.2762085640065238, 0.11465303582634406, -0.11926437388613724, 0.07153644942086144, -0.03125533103227787, -0.290060609191387, -0.4165194071025215, -0.21150534584413921, -0.3355835581747929, -0.03824982330137762, -0.4587774197700744, 0.10249123670777864, -0.16011709063304008, -0.2175606834954503, -0.4731319723206961, -0.362986634610287, -0.44731538387942815, -0.3845024612163905, -0.6018681645389409, -0.42889372768733586, -0.37180180202801905, -0.3409689444604904, -0.545291837312111, -0.6554712374520716, -0.810752565106627, -0.3015577986801303, -0.04535080625862878, -0.327184046086716, -0.4177269845953426, -0.4749005007573548, -1.9743015043127585, -1.2276504881171912, -1.3867445599131119, -1.3386659058924668, -1.134016165117984, -1.0286823047997118, -1.4731736173910195],
[2.6906496607882353, 1.8193042696179125, -3.2364363610267306, 1.2088246710523447, 1.3534777001051905, 2.0846626190750968, -0.5232387131218904, 0.32446998327769466, -0.6659688973677248, -2.0123405668171572, 0.2556187240742031, 1.264450056345974, -0.5011086347434267, 0.2078968070130599, 0.6290699014051049, -0.8109777384630067, -2.2241348272041885, -0.6099247423970626, -0.20237112991000425, -0.34218546212181544, -0.15813100204288405, -0.11431958413620759, -0.4862948159126468, -2.96264760410449, 1.2404549598331476, -0.6542617313641872, -1.0624947094757773, 0.5359277210212824, -1.6978045916816953, -1.3310508886709744, 2.5096770121628498, 1.559878681997236, -1.2811601897348368, -2.605647534848308, -0.5272626781312625, -1.9560366778969678, -1.708935072575533, -1.0609077894062882, 0.8544290157138521, -4.447129786045963, -4.838273994036848, 0.532457959593841],
[-0.15523455600562208, 0.05515439119803075, -0.05565897183724994, 0.2899286101075993, -0.0376264842359167, 0.1521146854649293, 0.1751695925629301, 0.10889557439929616, 0.06134004573478471, -0.07116162244178667, 0.07957991622796412, -0.40702055886069927, -0.10463143126884487, -0.03434726599993597, 0.18928588077523456, 0.5042203627676395, 0.5915879070789182, 0.31361328725215826, -0.42123391122595377, -0.139015150469488, -0.1752950371876285, -0.3433171796997383, -0.2027762536245341, 0.41619366814687175, 0.07662067988119209, -0.7004582443074051, -0.15926683218199603, -0.07789383922197467, -0.13825900759108412, -0.28931081883237925, 0.5841671295485146, -0.684026184584835, -0.7702462386043023, -0.47106272713710273, -0.20740235141946822, 0.027491363132726435, -0.3817437371846575, 0.9024641568666835, -0.9033414312980528, -1.0336511844775158, -0.3275228955419372, -0.6460305678546461],
[-0.18528832672807152, 0.010931107099394329, -0.11390408980021002, -0.09757806783444846, -0.20421404021541756, -0.07612967292366216, -0.09514459264276116, -0.1631327967673665, -0.04484219309164239, -0.04885702314184551, -0.09253491991470793, -0.17515215501734904, -0.0745550426862924, -0.16611243333264308, -0.14031954556113646, -0.33508106165645685, 0.05413009153996616, 0.010387589594252167, -0.34190870861129913, -0.23754381359330842, -0.17331959934317315, -0.37067745360770826, -0.1621010199719529, -0.30634621869838724, -0.46952666701568535, -0.3668470733187472, -0.2948410622189349, -0.33048408341973096, -0.7632234995255619, -0.5584955711359743, -0.5407017806205734, -0.18659051555740142, -0.03712455968471734, -0.3420080153739146, -0.20446415467011955, -1.2093354415948767, -1.1822598550746377, -1.4399158124037879, -2.20393648124552, -1.0500675304210583, -1.200553829853409, -0.6841528296856072],
[-0.31383145019021436, 0.006636434022185283, 0.11551358277755575, 0.09372743461454967, -0.008900874237616176, 0.20350416739814875, 0.02835463033859774, 0.04831333830726752, -0.13108854051317634, -0.3140663759021594, 0.0019007207498960548, -0.09459266143217057, -0.19676347491747312, 0.22087673901332758, -0.4259855035324858, -0.24037821771246973, -0.3398000887535245, -0.17979345291248125, -0.43442346239397245, -0.017774848584025674, 0.10258048767091975, -0.3209829369824987, -0.358289448572903, -0.4677173110082573, -0.29887823316001166, -0.3120005087199366, -0.48388513021911367, -0.26812184926559135, -0.6962482175951287, -0.7214681446363296, -0.23313858136290425, -0.5213309830390601, -0.4008606196788179, -0.34276274369882404, -0.6883042229379489, -2.1070054967471714, -1.9390909410376345, -2.166391687323815, -2.0015128212981805, -2.3063113161505293, -2.078873484055969, -2.576610414917845],
[0.24858820027426395, -0.03229510792201797, 0.06613229309615408, -0.30610559835608975, -0.21026962168854926, -0.2041197467358179, 0.05001321694500142, 0.06400640337795876, -0.032860542666952465, 0.28069402979984126, 0.3246276696820772, -0.2833377425825732, -0.10262124821960546, 0.002122698185823188, -0.002797061721734869, -0.30106360892211165, -0.41210148688461046, -0.4615785812083123, -0.6042140423465692, -0.3757454615871456, -0.4979636863825631, -0.30905592289975414, -0.5425921964003789, -0.2171323933609643, -0.3962012083514524, -1.0132099317443821, -0.7437922323571791, -0.1375321768044332, -0.36380991910986904, -0.8081649119391555, -1.1575684767240386, 0.3038486729487907, 0.1656941182793304, -0.6324913251710647, -0.6372401149564834, -1.665989288710998, -0.4547058363667633, 0.923728637986628, -1.4443054589278395, 0.007866554139576555, 0.3135651480854393, -0.6558574666922355],
[0.03249225955260438, -0.13925947785540377, -0.02376110966724406, 0.23765071502528087, -0.026269093136634926, -0.07495581736236516, -0.02302453821297352, -0.12919403233778964, -0.18473433635344522, 0.1729147224084195, 0.07478613793882316, -0.225945288869976, 0.14971660436546408, -0.015383388884311587, 0.007461832652635514, -0.051965054234731835, 0.022490423481969026, -0.1206647528930857, -0.20511455941888623, 0.0004644266011643722, -0.09217137532673969, -0.3414084296530644, 0.3219225962895291, 0.030800169688195787, -0.3615142804746356, -0.313269210730024, -0.33738504288881843, -0.008105616304178897, -0.7974144096953095, 0.7522106280015013, -0.5627723679759616, -0.2230398339071028, -0.06995079612233673, -0.2158015614711698, -0.12302266476198144, -0.17103096076602414, 0.0026593005278012696, -0.6982018161423854, -1.591328043108975, -0.5109054197993966, -0.9873188576383184, -0.4143763508413579],
[-0.11355251163193637, 0.0015029100289423328, -0.2388108315775066, 0.07898559704433024, 0.16496890207264067, 0.3213873635303642, -0.29852797374800577, -0.33611889418051205, -0.03643852623136744, -0.2928391059637518, -0.05349311788133852, 0.08950357252437158, 0.18429814642166162, -0.2242761501647261, -0.20891669514408986, -0.050228848874951854, -0.3076794638785995, -0.24874678471431555, -0.03655040067929438, 0.1205525669809887, -0.14249008610652913, -0.14946080299638978, -0.18061513699250253, -0.24688114186885887, -0.21441763421304555, -0.1945926117133728, -0.3542148381900815, -0.12439896780694833, -0.6876327048499685, -0.4316199739265763, -0.38104692011130015, -0.3276539524841119, -0.5000034627206088, -0.522867901626725, -0.7988823886232814, -2.5169295595679584, -2.350110353484338, -2.6171130347321845, -2.6898296252296063, -2.924434982441562, -2.471312064377728, -2.7152486853916926],
[-0.24847305265273795, 0.5478395485254351, 0.36358306752659225, -0.3043429940182337, -0.16622686423912064, -0.3927778578337023, 0.0027082852293415045, -0.24916801464307098, 0.2222783162248225, 0.09564445542569779, -0.09694971228636819, -0.05049792202151647, -0.3124596507213723, -0.0830370381563456, -0.024060744266540705, -0.21113527127080947, -0.09473080164645087, -0.055441305853998944, -0.3368720141002664, -0.2697080843852913, -0.03471545401978586, -0.13309471155606453, -0.22958356172423422, -0.05367612458525776, -0.30436865271220653, -0.19971504382596678, -0.4933035428299972, -0.11741602782765602, -0.911749617864806, -0.3519013559137894, -0.693833013340517, 0.03321495322083265, -0.0036259919962161186, -0.31962664514444183, -0.26547497582584045, -0.9097810143826511, -1.134495094109836, -1.5957185052390268, -2.4779346445085513, -0.6561259468680436, -1.0745074209972563, -0.807201213081272],
[0.01731875581661007, -0.06696590926834153, -0.07087654284649192, -0.22661299598850043, -0.0744999659090124, 0.0471752070760666, 0.07670916156732528, -0.17754624347826475, -0.10536105576825812, -0.12398923796640195, -0.15415405267395027, -0.20457434771657554, -0.045525844219440774, 0.13612727340570774, -0.33419524971638354, -0.0451961656634441, -0.03039982227069568, -0.16080656302449994, 0.05918163120763884, -0.2022010123100778, -0.0629424096829026, -0.32317264757210196, -0.46352302178039356, 0.06899359927356043, -0.1667725711022996, -0.10022724829082724, -0.1543804074000384, -0.301349575735494, -0.03693797120814312, -0.3588552369839379, -0.2729308378157327, 0.0895242766059951, -0.501064061029016, -0.5539427966354489, -0.15829812588902364, -0.20915893106904165, -0.6345547312889654, -0.31877431953407426, -1.505853680212299, -0.5642785401268514, 0.09675787248007614, -0.8449034940489987],
[-0.017699116898225745, 0.0312643072696428, 0.1455426805377664, 0.059557483282143164, -0.19979814998577475, 0.30277451109642667, -0.025915854515717946, -0.04728156149205246, -0.0724623191038128, 0.06815087759973153, 0.08979953558684331, -0.31455434099473895, 0.1394983043292122, 0.1998617321044307, -0.19883422176137766, -0.17475572618163565, -0.27414251572989956, -0.30460885535880244, -0.17471468324599873, -0.056311247025551486, -0.3234464136864025, -0.016141837342946636, -0.3832244238598919, -0.09275464784715727, -0.04860791219877174, -0.5462504424964032, -0.3537177032972824, 0.09353051563293849, -0.4970680982446194, -0.38781781135421967, -0.4631874394261838, -0.09488580229826948, -0.04893107100311255, -0.5169114832027448, -0.06736176439966106, -0.7636666921621112, -0.6876744763502501, -0.4359422845203622, -1.1258735483616529, -0.4789884664540819, -0.3018215803892248, -0.22255336424481628],
[-1.3586475203429302, -1.1079601659488136, 10.792595912263, 2.981459413748912, -1.2010516707562306, 3.152082297560419, 1.4737156617562457, -0.9994713941813345, 0.12016654624027173, 4.256026490523886, 1.3711194354287168, -0.15540261706134215, 1.2963285760472518, 0.15282761263140465, -0.9060099669136128, 0.024701587034694078, 1.1148239192523217, -0.3214057839559279, 2.306061036004914, 0.5577766634123532, 0.6359275794352174, 0.6320005253371588, -3.0149269078179937, 0.3619705882707812, -0.17057551684423275, 1.1732159592513998, -0.6534218976967817, -0.40251902827885655, 2.220540145241101, 0.02620515091970794, -1.34731578538963, -1.1410476527151778, -0.3158997246212539, -0.8843914164199604, -1.1567614231819812, 0.41852714069695157, -1.7767472896808456, 1.142130563279474, -4.827413454262503, -0.5050661669186056, -0.37421478905460415, -3.3484952740052027],
[-0.2380804970420826, -0.271260240218873, 0.3378944381034563, 0.0764652736061355, 0.30225979525847085, -0.3097353415464556, -0.2163494925169001, -0.21812412359241873, -0.7071526324742256, -0.18137964403281928, -0.022223228829319534, -0.44398146234940344, -0.7278702441224908, -0.20667724571635337, -0.11667260897683358, 0.3523117262001174, -0.5081299250036938, 0.6271095679947986, -0.3486810623486977, 0.33975975846745793, -0.42374983276779304, -0.7893722370750395, -0.9616168941670528, -0.3172325921510674, -0.8929305082373258, -0.6838370216253667, -0.8595330513987366, -0.7840425606910768, -0.7269004552914192, 0.15391460456724568, -0.298025300924623, -0.4586548868400052, -0.6359710659051035, -0.11737938280699153, -0.5757706104770627, -2.5960575999482396, -1.9803294265646667, -2.7708902902472228, -2.355498329228038, -2.494482538203502, -2.201064554933422, -2.7265596652812953],
[-0.34164177577452065, -0.022725721953792542, 0.11940908261647089, -0.03515394297257349, -0.20275476185745994, -0.07451988359561562, -0.062430504770326904, -0.22164187548095526, 0.04557077099405021, -0.11732376080087425, -0.17239163177164835, -0.17395067139125703, -0.2248536571047881, -0.0899268022221761, -0.19902247947335183, -0.2550275025974175, 0.06544717757219898, -0.31018915364854016, -0.16560034477018332, -0.2041014554000725, -0.1863101508691696, -0.10783029038605517, -0.3044597290121325, -0.08785836952259178, -0.32784956176771113, -0.16419610301625578, -0.2365575676351224, -0.208678812963222, -0.734856151759429, -0.35384273544632255, -0.4661954308832794, -0.1297800787276563, -0.3800189230863828, -0.6037031114436487, -0.7406769453951036, -1.7646418424535646, -1.7778332684161742, -2.048311261065155, -2.7893726508065715, -2.1545557456574267, -1.889630677516147, -2.1999211936782177],
[0.43397117240350364, -0.2127775461812054, -1.9573535736660337, 2.10083655881794, -0.0756170936017615, 1.2159033547091596, -0.08241087216248143, 0.7662432395348846, -0.42486455297817394, -1.4743287963794913, 0.7271377513562187, -0.39628918965895965, 0.15070519352899872, 0.535115422991396, 0.1260761950474609, -0.03706268271218793, -1.130076693822622, 0.47890789464325034, -0.4524455762007744, 0.26147173000396556, -0.5355318751858057, -0.46267405855865473, -0.4673948500712868, -0.004913283607161064, -0.8712886426188884, -0.6936528672162479, -0.5922459261760702, -0.26928075706839855, 0.3414620205256563, -0.4341734883158882, 1.8597078309202435, -0.8363550053965054, -0.41094343913702136, -0.7243983506443835, -0.8580500150084049, -1.6728964668469488, -1.8794163923670373, -2.471424190570802, 0.7094953329110955, -3.019997044354981, -2.5822178198650825, -2.7181375125858622],
[-0.17741501629214254, -0.0010112966860356927, -0.14885985619481157, -0.02430389470093108, 0.2779324669004907, -0.07677572628180071, 0.0774591607185247, 0.0780601733309031, -0.0888987611819064, -0.21668463861507561, -0.14717913934271498, -0.029328954310188157, -0.017986313123392467, 0.034584342396808194, 0.01369444024003909, -0.26602691492842107, -0.11897505120542361, -0.20844308055778465, -0.1288033508051604, -0.5192858514250913, -0.28953356792815016, 0.005405084088933769, 0.05872421821467875, -0.2720372853641893, -0.40383589974362905, -0.23696241137529642, -0.526274717119316, -0.16643762827186773, -0.39463502326985317, -0.48012912116200746, -0.3157729474501062, -0.2733939129172002, -0.5387507843786183, -0.4860751471300039, -0.6584081425070953, -0.8141969400478112, -1.16905819045306, -1.3715390528299396, -1.559174374673655, -1.496438205777711, -1.1938599244486605, -1.729129920467339],
[-0.12533891774942474, -0.26880413800743763, -0.3354121394356638, 0.04850256920652639, 0.07645507676446384, 0.15887203097360553, 0.14415986644805476, -0.07478856887467636, 0.1385816559797705, 0.23325717115302805, -0.020647141299575116, 0.14102470990275012, 0.12767509483324982, 0.021319967971088608, 0.2878742545317825, 0.1809102289151338, 0.18535905385088944, 0.24507554663004694, -0.08108314428533223, -0.03647271130631221, 0.006710100320536149, -0.22303878789810427, -0.4047937909380497, 0.18880129128148695, -0.1442293893383338, -0.42036967957144117, -0.41455007935950794, -0.26992391068742455, -0.39247481492095987, 0.0879339228366156, 0.6253811436570313, -0.5652177703278207, -0.6912154690117148, -0.6015853275636963, -0.5206741895875345, 0.3037394841343939, -0.5757846757963406, 0.27594319146384144, -1.0270897737614597, -1.9266955412926532, -1.354129521450521, -1.5478530929047505],
[0.4616444126098885, 0.2368376123327891, 0.050270765630080205, -0.22008169007522865, -0.28217083219705247, -0.31520304844252345, -0.37906462725056994, 0.14292454792110032, -0.05083994359396721, -0.073367132806019, -0.2104792239533831, -0.08846526225501326, -1.5858243956241578, -0.7237430302706283, -0.0315624765613811, -0.3338205415425489, -0.2605891217146163, 0.8477177686931916, -0.05762584381212834, 1.3171909061232354, -0.3633866768514088, -0.3543710478660312, -0.5382244412782594, -0.9015828074056931, -1.7982050661180606, -0.31631645726602964, -0.5564330012165967, -1.2576361666112665, -0.7853385281965645, -0.7791695987251062, -1.0946182353261613, -0.32051626276591755, -1.5236743765804592, -0.7808768490895467, -1.090478147310806, -2.931179452192442, -2.1368391831184725, -3.1191006865541833, -0.3528771567806516, -0.18113630638894027, -0.8885970874665509, 0.4578537765835941],
[-0.07224988419407896, -0.011771366931454233, -0.1270080266504735, -0.24388658299990917, 0.25353516019055905, -0.00679516808933736, 0.28290002711091455, -0.12603500590569122, -0.10257339281116173, -0.17614816518279378, -0.19649965398243932, 0.14974747839914163, -0.36287504110803026, 0.08237383254974227, -0.01960168248295849, 0.026138932259691446, -0.1854959667418699, -0.47624297922096337, -0.03529219534742621, -0.23239267199947153, -0.24048760011253978, -0.42330642378849354, -0.43236317844158556, -0.28479532646671957, -0.38229696746025177, -0.15845779864273551, -0.47258863757312347, -0.16889643942673377, -0.41128048564819797, -0.5119591825330649, -0.30450848469452596, -0.47669109627019235, -0.23389167185759085, -0.6725708221947609, -0.4114350129302848, -0.8190757824390719, -1.0211511292115427, -0.7221163369618638, -1.819689876984608, -0.48026070678572125, -0.8585804628069976, -1.2945281451624193],
[0.12613025142845577, -0.09587300465760565, 0.1020097324036671, -0.34750525540658245, -0.021747748786723566, 0.23043112543752092, 0.08752700768094172, -0.014948639852217588, -0.23249235620245373, -0.011338973321914345, -0.14785480948989532, -0.07466044914694901, 0.15391335117312416, -0.3033097598626641, -0.210903924614346, -0.1846757604063133, 0.11630311337310405, -0.525845410923046, -0.04172910165171981, -0.19166765222521145, -0.3296337308501333, -0.34014117258061083, -0.26733435878967166, -0.3986582706527617, -0.6733530002159176, -0.30647252359282634, -0.47229425736546526, -0.4866211857279565, -0.4908717845342252, -0.3980407932588851, -0.4185166042231524, -0.09715892184578077, -0.5365844234035214, -0.7983471270632798, -0.48792664017051784, -1.4660254146820728, -1.0394938780841936, -1.3751807332414363, -2.22601795321649, -1.3246616977245025, -0.9545429040600606, -1.494815247826616],
[-1.0884441432376533, 0.416119366946193, -0.0745939733840404, 0.7952682093947335, 0.44432432825649154, 0.606934040559021, -0.2959876084206475, -0.3099885311276243, -0.18390803388743932, -0.37547028287398315, 0.24765631968140098, -0.10870119608254356, 0.032006621704782766, -0.12856202927622234, -0.07282370797248354, -0.14261808177552873, -0.03877651037134231, 0.11635078836008006, -0.32081582438362544, -0.059303362265243825, -0.11619195251112192, -0.6897607916614, -0.7620588297338052, -0.41117634615402693, -0.14112506621555926, -0.3778136266753936, -0.20243077330519538, -0.12639009128193174, -1.0332636120957535, -1.3290663121674395, -0.4307072006455847, -1.1153266026864694, -0.5806161792639993, -0.7199875301597725, -0.8474762062489428, 1.299611463005575, -2.1292206746904734, -0.021956007767914506, -0.07289031712632911, -3.101299767253654, -2.330093515846261, -1.7203819190780725],
[-0.2447643216486926, -0.21292430903329634, -0.007612522858014558, 0.047840072695094854, 0.02428089231703144, 0.14559638284210447, -0.18971316186650783, 0.049507594479454915, -0.20118174795386895, -0.17308279612506886, -0.04249902229757975, -0.06222406692516222, 0.11961512804180968, -0.0633842641277993, -0.32074728122214163, 0.012911985838882591, -0.1152062082439968, 0.20115805488066305, -0.21276349066495417, -0.036776409108483274, -0.09236760969087675, -0.4102548120713483, -0.4326008015571972, -0.4877926069150842, -0.0651596057101149, -0.3839495203687447, -0.39513868776891, -0.41312143980464017, -0.4866277161391674, -0.2726657610673376, -0.5086843767149888, -0.6616763373207516, -0.4901982776258008, -0.5219447800945906, -0.6376145727462814, -2.0466651457788267, -1.7169809873552975, -2.1277716464249106, -2.1040026405058607, -2.359038459906109, -1.9490304005207277, -1.987773883329928],
[0.21479482440033135, -0.755117466308469, 0.1454667692867217, -0.23891467025340768, 0.3253783883034401, 0.15168183832389412, 0.3194098450176, -0.18378297032149163, -0.303595492052017, 0.030098417296486695, -0.6233388303646766, 0.2954017510307208, -0.04953995250129445, 0.12183021560574284, -0.08194332122927929, -0.24019991695356405, 0.2653490499976205, -0.43228384630321726, 0.2846332411386825, -0.1456765415721638, 0.1487328120817676, -0.32909612299757535, -0.957525951516939, -0.1462095550133198, 0.1275163742693942, -0.1687091139774546, -0.2956994810114776, -0.10344934911827271, -0.6764774765693893, -0.07551714890162146, -0.10726541923541406, -0.48625702016975875, -0.6431584114016038, -0.671942601831511, -0.5425614051608967, -1.862702840734406, -1.4070509687292612, -1.5641607111466733, -1.8211132980704687, -1.6489890973313572, -1.8814851656795781, -2.159393100266105],
[0.320641786483942, 0.11049805793303283, 0.3927101635660928, -0.10367450091182302, -0.41493945816827044, -0.3958495562713948, 1.0719749285915325, -0.33213778066776195, -0.07358362402608294, 0.27611492358447337, -0.3335222794202557, -0.5358826293891591, -0.4191490080135323, 0.6923902590593946, -0.10595485042299532, -0.0608884095056448, 0.3934290298395543, -0.3501629697893428, -0.5234815645267089, 0.15842708998861302, 0.030169283125050444, -0.6857145796054291, -0.4384574072942729, 0.04680066048932295, -0.5949866588412936, -0.5365805963188975, -0.703570420480263, -0.5171247124761041, -0.8559474674420978, -0.43632664877051447, -0.4574170778041698, -0.5623416832441698, -0.2779903248156727, -0.39705352746791533, -0.4017658683453386, -3.050712031150226, -2.420079377779362, -2.8763896451613196, -3.049150544193275, -2.5809995804946193, -2.6081002443139893, -2.7779014497765293],
[-0.6243436293237395, -0.2287491500700242, 0.3615616503495008, 0.1659693494222461, 0.12337204760252025, 0.15703214758290343, 0.25742105991319403, -0.2404934773667556, 0.08276022380555394, 0.04164901150625549, 0.0979809537608748, 0.01747207043093237, -0.01819806831105095, -0.10471717425397208, -0.15085223501507508, -0.11810147852625513, -0.06415615897938137, -0.11799245386532592, 0.090085385347669, -0.29713802456014865, 0.14582176628832538, -0.05935854594907545, -0.1897351803723506, -0.16916923299884168, -0.17203723286717154, -0.12436435236826979, -0.6508313873931701, -0.32901184598105004, -0.9105606739571697, -0.2884873834959753, -0.6541734759471346, -0.2409713852732024, -0.6289760314916705, -1.0439640769195189, -0.867822977903978, -3.4409928869269213, -3.111191396267277, -3.3566785990671257, -3.6422379306823536, -3.36634589077889, -3.309869465661817, -3.499940439675072],
[0.018458524287201274, -0.04344627951791615, 0.08390386044219797, 0.1685729484050059, 0.10239530197647626, -0.10442908184303212, -0.04162517963178325, -0.2045122078604551, -0.0949538706673419, -0.0111959608822387, -0.1380470492614279, -0.042376270350946425, -0.39482471946089115, -0.3630220785454867, -0.29149078767392683, -0.5664992750282221, 0.21111356676541618, 0.06313874679006007, -0.11572861073018935, -0.09476872524766385, 0.13791528408339437, -0.623077832853755, -0.5251128404971592, -0.4505636762371649, -0.24747553220673685, -0.02144265759490189, -0.6188954902775582, -0.30911861716822736, -0.7583983614064368, -0.3759776830047086, -0.841288077146936, -0.3348070252258144, -0.4099153144365556, -0.2922531397717929, -0.7483189143699696, -2.0823995770171835, -1.7935737993702172, -1.9792539323937788, -2.209269254595211, -2.0433334031822787, -1.7541134355950805, -1.9607318366525077],
[0.6352572814052331, 1.6406810440538042, 0.010168091259674472, -0.17058054725080887, -1.467896445871588, -0.9048590442385832, -1.0017583417939755, -0.7689229132389628, 0.5821219406974634, -0.6580200636783523, 0.15480041884076962, -0.9483088577020489, -0.5351766307844629, -2.950985156594982, 0.07305369850864131, 1.1778534734209345, -1.0941981093557747, 1.636667991025546, -0.9125538312529061, 0.14558204645280076, 0.21412171747526654, -0.3441481022705497, 0.7012019690748995, -1.6598116172201938, 0.9691994797159557, -1.1317946746755825, 1.01041089423457, 0.5477030346114048, -0.6584102827453067, 1.480855430149754, -2.4787492173494092, 1.0472845432144835, -1.909214096885267, 0.20317294182490397, -0.9604681479258496, -1.334855449591754, -1.544000240796588, -5.718735534407366, -2.194039195822961, -1.467380795249928, -1.522215117904382, -0.13872452605658223],
[0.2522107379268718, -0.06830639034263418, 0.045879445743897795, 0.21663571347009303, 0.2599179150966239, 0.038840647512016345, 0.18012537982619098, 0.145676508319605, -0.11372275814085055, 0.0639280140850798, -0.17519554196819087, -0.06653367883670445, -0.29518918826561236, -0.09574718368623693, 0.051024757974864, 0.26441968560165985, 0.22986922541924393, 0.14545709175690724, -0.06490795883943289, -0.29247238802293696, 0.14663328005182463, -0.646679052728803, -0.3726935271275484, 0.04536469505952974, -0.4943951306356961, -0.1921290416628476, -0.591478251999904, -0.2505266294985481, -0.8302356836005412, -0.2153952563347125, -0.5832494772416793, -0.5381502744725246, -0.7819167953913079, -0.23654143893678176, -0.6973710357037798, -1.9724383873644173, -1.381242218905914, -1.614203799588933, -1.8273804900049557, -1.6557477944711634, -1.5955706919817016, -1.8722014512720637],
[0.33066956211718807, -0.33089605721550436, 0.10290484658804382, -0.41421150009241653, -0.06228924327416209, 0.05422759469255984, 0.006799668299350474, 0.037842341076598895, -0.2700227238791227, -0.13435035055084305, -0.18414778617879207, -0.013759733532837213, -0.2772310412410814, -0.5509917946063893, 0.17184390504686634, -0.5928581926630113, -0.28572326191025454, -0.4899110241761782, 0.48210595427315084, -0.5144802649252049, -0.24798168820343572, -0.3443062398112854, -0.21820053499412087, -0.40817614195638485, -0.8085040730925084, -0.09692876735903658, -0.3134967909361945, -0.25067600764523973, -0.21887088381414094, -0.9439483847002095, -0.6248358838110674, 0.1055647400627854, 0.11308529603156621, -1.1020625715788381, -0.2554245477271358, -1.3962966939985206, -0.9360996945481761, -1.1730610514924174, -1.5906637118719713, -0.7569826048233984, -1.3288265849893974, -0.3104479206342004],
[-0.07135723883313377, 0.02200467036805457, -0.1374534852679252, -0.17612133643955513, -0.18781331210510455, -0.013033791675091989, 0.12719336352099517, -0.20004225418049207, 6.846811609631868e-05, -0.09937584460743572, -0.22599339413934913, -0.017106268494370157, 0.04147907653465735, 0.08712164509825919, -0.08136997499813345, -0.4674993353276755, -0.2823494820182356, -0.5220884856110832, -0.08743137958801614, -0.6224934880158605, -0.1349671778028465, -0.04025702970870685, 0.005597088255908671, -0.16840894047565574, -0.4671423486921417, -0.12894761150333509, 0.07259954046361693, 0.0716713414869722, 0.1430264926286361, -0.5903835880118523, -0.20256699476749224, -0.04421855269357378, -0.519423716987974, -0.6111616727057837, -0.25704653346341444, -0.4171066365422621, 0.019471642503272497, -0.8024708602459301, -1.2901479804262825, -1.2559793167359672, -0.021015058952185587, -1.27269396344057],
[0.3405319043344316, -0.7669015772472194, 1.648788359550614, 1.0411053435986097, 0.7388696747899318, -0.2452538319911731, 0.7049891368793203, -0.3072296782153791, -0.26878763161706415, 1.5237981529237234, 0.14891772372645964, 0.4705376943552426, 0.1612449834690085, 0.5603749970630734, 0.16458733537546238, -0.30109702243401404, 0.8319466243992396, 0.648197681973462, -0.3101718520443787, -0.7266761269075686, 0.1367018737690604, -1.013389816296129, 0.2110123574147983, 1.2917694438732734, -0.07792222011944709, -0.8735805867694012, -0.3768664903240266, 0.5334694094130792, -0.8973155972903091, 0.9067376166852683, 0.6156780581638053, 0.3366765859648918, -1.7340408735606612, -0.157295965445297, -0.08293569757761583, -2.472960037763719, -1.2561190881545965, 0.6287251481491376, -1.6993980543265401, -1.8751665695619504, -0.15902939243501543, -1.5739872634278258],
[-0.12218120464485165, 0.2361532835741048, -0.20623457218700794, 0.250270623084235, 0.23665872557458406, 0.09740440358776682, -0.012371562934583353, -0.07268600299516795, 0.07073793738445175, -0.23624861603829403, -0.14278091393368408, 0.20230716132456483, -0.07034999922902474, 0.04182255538809786, -0.0013939586315324078, -0.14666969351024758, -0.35663627305484064, 0.30664304081905797, -0.06442263535376054, 0.04718275592130596, -0.14329258289954025, 0.12927118400026943, -0.18111667473182605, -0.26856155001008203, -0.1315826631971016, -0.1886062237733094, -0.38336673421006046, -0.24493063036073554, 0.2941968430016876, -0.7004275176453089, 0.3888183528577844, -0.4292309344768473, -0.4794421710943464, -0.36208109290776036, -0.6940307797305332, 0.6198161129393092, -0.937130782939485, -1.0391204880030869, 0.35649308127548224, -1.7074273577113577, -1.2412363343586688, -1.516731659550771],
[-0.0759438855420979, -0.07593876296338661, -0.26321655213981415, -0.07807511853719563, 0.04221743991103158, 0.09078001067783886, 0.06340410813164096, 0.09101986440291006, 0.02158160160235375, -0.28804218435576395, -0.05010211839094175, -0.07525554283914347, -0.1261027176841924, -0.1996660014033618, -0.2795275204064271, -0.16912585896287588, -0.32601685057317753, -0.18618933160349688, -0.35370723722676484, -0.4881513944203865, -0.2530393000320914, 0.0893999420015175, -0.30435552857573644, -0.35564213442125114, -0.28318289075661457, -0.7353088719848592, -0.2292379975213791, -0.10621126747879288, -0.10010337915956678, -0.7078972780055409, -0.12132495420635628, -0.3655058748786788, -0.6689767085124815, -0.32103077868351854, -0.6885093889064952, -0.1752087616915139, -0.9511863199214893, -0.8581307849982843, -1.1961321418640378, -1.217663011852451, 0.5529884717478923, -1.5245834349419236],
[0.39953454392882604, -0.6718401831385049, -0.09331473053970754, -0.2961272518098951, 0.30086917233533106, 0.4646296999765129, -0.1233757414900408, 0.23254402888732217, -0.07450874144416134, -0.18550029629028045, -0.5069247915056982, 0.08207259526934493, 0.3746430435092982, 0.21140065460997104, -0.2600719698508107, -0.06400159160056773, -0.31088268851703615, -0.6029041390662966, -0.0689811318005073, -0.1710460667750562, 0.26237861112911764, -0.07827561251185691, -0.3623806147138076, 0.2521055520685848, -0.17545888313284516, -0.14686145648331053, 0.026657303401714558, -0.06439355098282341, 0.031994273550611914, -0.33721892984806, -0.13505955222806953, -0.22920400080836037, -0.7995491490051838, -0.7829472196506051, -0.509177864125768, -1.0833152390423897, -0.9168832188623837, -0.9868345494885912, -1.792246157274975, -1.7060361385204097, -0.3064388946819069, -1.4529128210375852],
[1.1361090696611005, 0.4999129095611814, 0.763799691262776, -0.4604654817769939, -1.607703543459081, -0.08327052521945767, 1.342719928234904, 0.42632040192011933, 0.3648329889877824, 0.311887342477309, -0.4430955869099499, -1.121327460038396, -0.05513368401971524, 0.6746714188738907, 0.4540014260057805, -0.21808664123042182, 0.21131371293898008, -0.06457181436661176, -1.2797088964096943, -0.4819790764895687, -0.02525838939163807, -0.4797131999850357, -0.398102216647602, -0.05880279945005928, 0.25095928837995585, -0.279099004644244, -0.5437628790369538, -0.5239598261630657, -0.9292886356548045, -1.117313271129089, -0.4389599811320046, 0.6435424144346488, 0.9745698951990962, -1.1647208388396406, -0.9276898716141606, -2.574808362629561, -1.2771886700938504, -1.3328433000269635, -2.7011902644712062, 0.44081255741896774, -0.791282636053984, -1.6329268524710079],
[0.22314988564931337, -0.0038180597574730647, 0.050539690224586875, 0.11159801335542374, 0.09145207330852388, -0.011135050322231483, -0.19223987844798368, 0.14000208921060353, -0.22562231448574352, 0.08611648931875754, 0.04655573391236142, 0.057443654243343883, -0.017671968751075684, -0.1764809056378199, -0.15089161164282314, -0.3363599762703038, -0.2266357568493735, 0.19876812196519086, -0.2874461353168135, 0.11234155696975737, -0.3294987401807617, -0.3062911521394017, -0.44792320096467264, -0.16602865300456132, -0.05523492777838085, -0.41939925183466775, -0.17533986376739544, -0.24451678120042805, -0.7561080923444045, -0.34160752512664055, -0.39087379918022674, -0.4792989718655651, -0.5306390440232137, -0.2332599734114222, -0.8314261852021294, -3.0600430729055375, -2.6935734139769107, -3.066069195409875, -3.174480925664597, -3.260417130532943, -2.976802629472826, -2.9396925439521464],
[0.21287145418187198, 0.46158544392476175, -0.5984749458695492, -2.3720267132067905, 2.2167758405676574, -0.7490977576952773, 0.6265161562759529, -0.44313850117933135, -0.19433462114555358, -0.9080911014066179, -5.124589911764686, 0.8035272287133514, -0.48442673303857403, -0.5495978321905093, 0.7395386339475725, -1.5701315566068965, -0.743032036192091, -3.068443039113287, 1.485307504716114, -2.503809536778067, 0.31483979198251805, 0.23415185092890262, -0.7816443857601705, -1.2227161359292786, -3.3116125537697014, 0.7345111567228703, 0.44449260237375765, 0.15211932097517966, -0.7366774020537975, -0.8872839470162477, -2.6140065934840044, -0.23072416953221797, 3.952264126342359, -0.7082731972389705, -0.48191858528905274, -1.0941840233589502, -1.4146132971901042, -2.3997392930078227, 1.9091022464580831, -0.7507122142708802, -0.9731479235430425, -2.059390891849653],
[1.6595492743092377, 1.7687839153764664, 1.9312023063785273, 2.6945216632745588, 1.2720309271401231, -1.6075085108168419, -3.659226529784739, 0.19002214978494797, 0.8350639846985667, 0.46727495229024896, -0.36786548505409994, 1.5200954344258024, -0.3534505672347843, -1.9267788402403423, 0.2819420734645298, 0.16436291273075304, 1.036915682777389, -0.24470122500975117, -0.049815961910180714, -0.45042445054286095, -1.0601817572101158, -0.32448276153332145, -0.3344719976067407, -0.5391844531004714, -0.932217465364953, -0.19167938159608333, -0.18998459427335646, 0.5104153789244931, -1.137877553256963, -0.8524890756619581, -0.8093410593706499, -1.4559784775527458, -0.9451623100832042, 1.6504873521300205, 0.37882841262424605, -2.7396483419389446, -1.87256380709541, -3.4509110647943597, -3.41807178486576, -2.6588167291581795, -1.6894656206106637, -2.658703848545686],
[0.08708468788778484, 0.1698759594516943, 0.20499430713654374, -0.28952670225634525, -0.06662800380933899, -0.08378989839484309, -0.0926773274236706, -0.0019353589757912235, -0.08862988159413267, 0.011098379833097024, -0.23361432962693088, -0.17652578647297357, -0.1553409903615857, -0.36708949162050813, -0.14666931794668742, -0.05791321648439225, 0.038076097756763544, -0.0984677951806865, 0.014359969556651659, -0.20123379700877753, -0.11704568423982804, -0.3123152794072049, -0.34571770540110136, -0.2228190491491161, -0.42213090736699477, -0.3462667893641174, -0.17494813981810575, -0.16782099524148633, -0.7954116710885156, -0.5858635292504197, -0.47447497451080173, -0.27929389959264284, -0.5521771258469764, -0.5252056078721598, -0.6151717080338499, -2.4155910934495823, -2.0662412910595127, -2.412612717205855, -2.825611838901997, -2.2718908865092615, -2.08857167359972, -2.22224228858668],
[-0.19935709405912022, 0.07640336324861878, -0.3236452815627433, -0.10572380072599373, 0.23388432838640139, 0.15055530985660945, -0.09867511258492745, -0.005487228412927086, -0.28226007429025285, -0.38368347608910625, -0.37921335081469776, 0.14902902673005927, 0.10429786729418787, -0.2841165060858658, 0.05772457677351259, -0.03910613892768877, -0.40765062801357105, -0.48060987797856347, -0.08817022948651226, -0.21639107754238515, -0.15567787800257488, -0.16854647305455467, -0.15340312118451657, -0.42390996445426093, -0.23389491642649562, -0.09884378411274154, -0.13367683303981653, -0.2693173714792824, 0.1155202763394695, -0.48239755630364334, -0.14090475430502142, -0.012299367469573485, -0.2632522298295355, -0.16670917278624747, -0.7325751988237149, 0.3523089255461514, -0.7008816301192201, -0.3940568830454826, -1.2281427356773413, -0.9230925050565518, -0.4922766159701776, -1.051561165014934],
[0.6463052130445367, -0.29239774539173224, 0.8890405588394996, 0.2184561786657121, -0.5317139857942201, -0.0945151831068052, 0.5235574601913567, 0.3601640186445946, -0.24799662051499632, 0.4708729306414244, 0.07562245722296307, -0.2933183053058008, -0.09602222035116209, 0.44335762906378057, -0.034765089301484316, -0.05189466929316249, -0.08612632798278387, 0.16857692252236203, -0.1034408210770582, -0.14408067028297306, 0.07581763466493995, -0.42506315381006543, -0.8642804038388995, -0.07570372980483711, -0.32219681300311415, -0.34374895178445003, -0.31411322599104213, -0.3879953861226885, -0.638815371195337, -0.36242845935326534, -0.24109143781279543, -0.49958263331419395, -0.17072376256207497, -0.3936021317136997, -0.404246551083727, -2.481180644474106, -1.7359618856436532, -1.7193061775600713, -2.1787046073139087, -1.8041772449855826, -1.7074232926794126, -2.036971930925304],
[0.029839322672712554, -0.10099492600506317, -0.3342447297471556, 0.03612846944386994, -0.14262528992700427, -0.03178159284805331, 0.28668314730401495, -0.07625682338793974, -0.1555035507167592, -0.2972164913525592, 0.033530282046925874, 0.08648373391537605, -0.16878621040270383, 0.22556824762443733, -0.15156449154340532, -0.1817654829524055, -0.003016751922627093, 0.05177188704217639, -0.25054452527398796, -0.12898760395357128, -0.08696050726589091, -0.0699821950567491, -0.1445012605557269, -0.08291901824123117, -0.2567620144252466, -0.367779962722908, -0.4377687206656611, -0.22545425395449206, -0.18443064233776335, -0.3851784824594961, -0.07524713388751733, -0.47181067540096405, -0.07522932362470788, -0.46318866984365226, -0.6994331060040881, -0.7134467712260022, -0.7731660616209336, -0.646204362416282, -0.7604526871220839, -0.9248266062617786, -1.1533827684391391, -1.2998953277890752],
[0.332872033889907, -0.6196149053665385, 0.9177531829785249, 23.718460171641333, 0.31762947768972905, -0.22819224482997585, -0.5973391348535214, 0.6361317798610076, 0.7735060337243218, -0.16206751270704725, 0.6920852594992245, -0.3813893811990799, -0.3962326643332181, 0.04233871947836156, -1.490070685254371, 0.11952539119554521, 0.30746800999478013, -0.0032477405784029916, 0.20729137098701966, 0.1253137263914654, -0.5169760906306148, 1.592910509338216, 0.11261909795201047, -0.942341169970906, -0.8324000718686462, -0.19939092701276576, 0.5691552979113039, 1.70111203483515, 0.8105069511075966, 1.2218926794165883, 1.6592783136552502, -3.3550048621522754, 0.7241883610523473, 1.5087552838703873, -0.5474932619872707, -0.2302967623853459, -0.9287237128993763, 0.46904751828858604, -8.685337430336062, 0.5246774835102425, 0.5667856835156515, -0.020865438842426084],
[-0.07394149522024834, -0.046062766988554535, -0.027450986484314054, -0.18259313335998292, -0.10944664676123375, 0.05472181135673667, 0.11091052154218638, -0.06077757793036414, -0.27856413635440747, -0.28757071496741216, -0.2833154108909698, -0.25178139162940355, 0.06299878876513157, -0.05544992976118462, -0.3435006942094993, -0.3590148702101198, -0.5325076033711111, -0.30560681816733587, -0.4932765408472001, -0.2890044396233514, -0.26727038773396294, -0.41578612615193483, -0.2587566774371773, -0.36505477708449696, -0.35915262302061746, -0.43149575290633496, -0.32311083139199953, -0.055701339435426545, -0.33981733862545077, -0.7451044876245216, -0.3088257873154909, -0.06951863574511052, -0.21073449795779625, -0.7690303072148607, -0.48163149937009175, -1.4497203510947614, -0.8424545160894846, -0.816638652083874, -1.3892262811633824, -0.9993368527442239, -1.0638887495874823, -1.0751190490341924],
[0.43577551196179753, -2.3979938479765113, -0.9182895239263479, 1.241997843588757, 0.3845394816627377, 0.7881661576417223, 3.0891376693251735, 2.055805003973369, -0.5881241734089441, -0.6286991523903596, 0.7406212258754068, 0.2949919076360488, -0.12167989521966728, 1.8087472084753318, 0.7319440075086359, -1.0627515914071424, 0.1920332885610735, 0.3620223915441486, 0.03945328051794157, -0.5004621113431017, 0.7714267569675032, 1.0305910183658558, -2.0257628376803187, -0.7932704504481747, -0.5531727388567159, -0.10353788955714889, -0.4196084350800839, 0.1372456536060886, 0.6689781481603944, 0.9764365897628513, -0.595464934343988, -0.27736562729174125, -0.7921137832115503, -1.2150956215230546, -0.42024828944651327, -1.1045550142981315, -0.16482950420682752, -1.0776269105336018, -1.8238816767181647, -2.1091112651622526, -3.285877315217258, -2.2676772479705716],
[0.026650488641018225, -0.025478528235284948, -0.342320046983583, -0.6107133455250106, 0.10257856368498819, 0.16857890631465036, 0.16914876796939465, 0.0466822983617506, 0.17689074850168693, -0.2126223934287503, -0.5553306048504922, 0.3002464090608719, -0.08499100650761918, -0.0029309423567224252, 0.1561655086238773, -0.08642099771054675, -0.21209380824929586, -0.6532129678317693, 0.21643390787926614, -0.8481283679338835, -0.14439388573555004, 0.0052674196824612765, -0.040125036596474144, -0.37392277158637005, -0.3820905822113783, -0.08640887355412301, -0.08393725648080201, -0.10200219232239835, -0.4391201359308898, -0.7946526236952064, -0.6486551412300581, -0.07107834250191652, -0.6518948712044715, -0.9840592680288044, -0.7891679708529767, -2.655732051888492, -2.2767071239235595, -2.487320781138092, -2.68337325879744, -2.7277591404216004, -2.2028871487883404, -2.850742225128453],
[-0.1054045167172385, 0.10933480737496072, -0.01595144784495896, 0.4261777856371196, 0.22610421094108232, -0.046069841447684744, -0.003762374459760052, 0.01938585083992251, 0.09929051301226997, -0.049264408635050494, 0.38662682577926283, -0.15117388897172254, -0.04210887090141559, -0.20719983304666037, -0.007227317572358165, 0.19174436787943566, -0.059565672543209144, 0.6232161813836359, -0.17319247618998376, -0.00048420224943773217, 0.07887231230369986, 0.016704619672928474, 0.21063238867504616, -0.24010492402130093, 0.2964988137463378, -0.15329148365752815, -0.3849045096666028, 0.003336734724765399, -0.8645872213232523, -0.34469644197218613, -0.15571180442163465, -0.05704088721397191, -0.47938238435946456, 0.1628273920234276, -0.20674779309207797, 0.6262787672520098, -0.31656739059638794, -0.1907289778465434, 0.1394463317119815, -1.1513201688520345, -0.4219920739831253, -0.7469716640615658],
[0.06377186316996204, -0.09489701888144576, 0.022948987691701065, -0.017191851604704, -0.15765478584767842, 0.20707089744776486, 0.006239927330358752, 0.0862668695266647, 0.11422533167381316, -0.26481181844623924, -0.05333798165078011, 0.03672464381805567, 0.054000946080003144, -0.10798535076450005, -0.2267599508074413, -0.1421219221835194, -0.10177062130072011, -0.10662190150806898, -0.07723136335526606, -0.11482886155474735, -0.12512824902165423, -0.188634302140006, -0.2540513407567399, -0.220924890212038, -0.22952081771025112, -0.37322444484791717, -0.3310300310906019, -0.13533536711597066, -0.3470670123691399, -0.25037268818455893, -0.3306438956213629, -0.2772582763961904, -0.2851707761282552, -0.44686543317048294, -0.4434908017006129, -0.7291135272987493, -0.9310103570243272, -0.90724503638108, -1.415270260465349, -1.2716676204233148, -1.1042561402560804, -1.1035553671593106],
[-0.05240610077027, 0.006776290642132735, -0.022928734496429556, 0.04952222435056358, 0.12193763400545325, 0.023663946724326382, -0.02054757636725107, -0.1377068494181351, -0.3761677128444523, 0.029142099165261334, 0.046467260979716415, -0.17114748142942277, -0.41177128877220165, 0.0626346808615993, -0.30632400466419146, -0.23099414654202136, -0.30991768153169696, -0.12589377003211696, -0.40685385774262717, -0.05381907312842167, -0.17376739426678145, -0.2672273453555462, -0.3192096500078638, -0.21712537307515217, -0.5617033030509153, -0.39086765633800474, -0.5741893765263525, -0.43330463499754296, -0.7110587198928958, -0.3837141297156523, -0.5652533280687628, -0.457912443033053, -0.496804201270893, -0.47888299365917014, -0.45061065771243203, -1.9333413934543713, -1.5404549309494404, -1.8679807120999108, -1.935143212905222, -1.894531495729842, -1.7682692180107542, -1.8812191876412783],
[-0.131630339881153, 0.0636735461124208, 0.17417972853654376, -0.1904345799626809, 0.1746547516084901, -0.05548695804479098, 0.18771201229859494, 0.015585814517572757, -0.08061738437383785, -0.2668544055080495, -0.14713965524611008, 0.23189604120791504, -0.1000320908335661, 0.030781391155746907, -0.0882665330777396, 0.03716243545811135, -0.2015115946639188, -0.4538664052037854, -0.4214004258675569, -0.32455150921769865, -0.09803105337183625, -0.3271927589451549, -0.006433958789007647, -0.12715459373648144, -0.372545902164082, -0.004960766955588529, -0.6078064411083989, -0.4483212736493941, -0.3866103161673054, -0.054697700947372925, -0.3017706423667926, -0.22315063260872073, -0.5804526574034579, -0.4382526065056544, -0.5678299401265803, -0.47328188428404905, -0.683274975615377, -0.8583205667660392, -1.258112388693908, -1.074063361061584, -0.6816746468449175, -1.415657796398806],
[0.4023097397172695, 0.15169025960005547, -0.17984283686864322, 0.7498555035374717, 0.14339329620380784, 0.21169522638085525, -0.23010879332856168, 0.30773260077320713, -0.35814624837025066, -0.6707706669472163, 0.4482737805829364, -0.24404233974301842, -0.07682150990150276, -0.40303051517583177, -0.48793084057363156, -0.2016743833074, -0.5814664547743197, 0.38491381892224014, -0.39796221547691946, -0.08838618182837088, -0.07312195384929167, -0.46365337707073057, -0.657075755048477, -0.6422394806588662, -0.2809058321170481, -0.3941843938039469, -0.30229665299208786, -0.5632334304349004, -0.1840910041713123, -0.45000504513076195, -0.2908554851584209, -0.7773683487476614, -0.6813902058279285, -0.021416255678897645, -0.7280475983425926, -2.2221182434018285, -1.8167593967795548, -2.1836922180732024, -0.8967059273105381, -2.499078010672022, -2.202063858239386, -2.0838048003973366],
[0.014589726390503469, -0.04132180866079765, -0.07108866588675174, -0.3024046365348737, 0.03598046386358937, -0.048040530718439965, -0.03115157418580136, -0.2113609921239614, 0.0482631270445689, -0.2861893022968044, -0.19465407097486348, 0.02797310547853252, -0.1939747221059624, 0.07535447443011707, -0.24931534692509627, -0.09376281415181885, 0.0464437605973486, -0.23896183398622217, -0.007971329691046436, -0.39657629844282494, -0.07583748861498348, 0.15586855793890786, -0.17024052432104209, -0.12708257326774755, -0.4584202152464105, -0.07081817747989601, -0.1212183167085484, -0.21819384120772758, -0.012591526582832394, -0.24298658553723823, 0.1865500717845957, 0.06100660036162167, -0.3512416746255542, -0.4226562853761144, -0.2437712188728309, -0.23550601860126272, -0.19639918083241478, -0.015395425111203532, -1.0001343972858994, -0.39598964727445946, -0.15923303037763917, -0.9588442497471384],
[0.2653570185110705, -0.2810525118571884, -0.18275880645814385, -0.0987138000290128, -0.27741921697545163, 0.08739911629004249, 0.12471259168373908, 0.05132880626050797, 0.2776679570780964, -0.3437026682645962, -0.47085334833457004, -0.006555200300853788, -0.4560367871971341, -0.07759418129748724, -0.1560226977147686, 0.3833288533923859, -1.5509138172168657, -2.0026235173453593, -1.5853116902382165, -0.4167257401510099, -0.561398196667099, 0.08871734307726804, 0.6541102372185352, 1.6648635051230198, -0.1490843443630337, -2.087185368732456, -0.8688753995464197, 0.15081633578375772, -0.18966170852527378, -0.6276150706491694, -0.7617694343814968, 2.386487620869468, -0.04294797123442689, -1.7820217571124268, -0.35363911940401044, -2.483100478489495, -1.686053149711563, -0.9534782941892265, -0.3611790585321281, -1.0013728322343352, 0.6753182820410872, -0.4899637208527014],
[-0.011737980203250908, 0.1263801656324091, 0.12897294946951515, -0.36222436809340586, 0.031999505346582975, -0.006722137231947588, -0.025234013553117444, 0.05945935909703917, -0.03851431788933224, -0.14810060700334793, -0.28433389280016863, 0.011148213531751052, -0.12887727691223097, 0.005947938019135766, -0.0914699380289105, -0.1841441195053356, 0.15092009589707342, -0.09813867827976823, -0.17543636345984043, -0.1960142721423405, -0.1773992133430871, -0.247340359345965, -0.24995126861235167, -0.3583457116386807, -0.3141771994338423, -0.35121256079669483, 0.050070629188299604, -0.22697423127353836, -0.8853078724711285, -0.6008302111925946, -0.5445210193188731, -0.2552767180799751, -0.38535199081113314, -0.4606818868492627, -0.7243697206319676, -2.6687282806292156, -2.3415797474858104, -2.6330846229416185, -3.0160132253922636, -2.700112873924695, -2.3925944135088915, -2.567961519232188],
[0.017422996320901944, -0.20228984525707444, -0.05646071990946819, -0.09633274000223215, -0.016694989416274963, 0.012521003943304231, -0.030338105292120127, -0.21662659457290398, -0.2938003514877998, -0.12749764708335776, -0.21614183812430238, -0.13180956828385382, 0.013031225500546945, 0.17562266792235748, -0.03536390399051604, -0.11407772628622344, 0.05670031464804352, -0.17561199683625658, 0.06462728857844875, -0.2772267228324157, -0.20804073729159783, -0.4405228436265862, -0.4269964689180384, -0.14391141460069384, -0.35894271440659636, -0.17213066079371628, -0.13546355382361996, -0.18171557010583686, -0.6648045230227686, -0.286219545336264, -0.4322275131152795, -0.39033036739586713, -0.45433790954211245, -0.6792907802700768, -0.8140327340688321, -1.836489892493591, -1.5980306773941078, -1.5514569853713045, -2.453470843293111, -2.00998889921481, -1.8453723254750451, -1.9406370404111464],
[0.029984638367632172, -0.03408490510113269, -0.12549698271330378, -0.3016955609957721, 0.02640927761036435, 0.010429239977310423, 0.13448818451049852, 0.2155304141495626, -0.29270887685957603, -0.34838440413979127, -0.2558596891022908, 0.3741130562684643, -0.0032040060918922817, -0.08676902765210788, -0.06904037657995572, -0.2706331131380826, -0.31005939256346027, -0.42601799238297433, -0.09538276505596498, -0.2969757693505143, -0.21599484889061663, -0.30337454141446213, -0.25346244000489404, -0.4881198564743499, -0.38567844286446246, -0.4037509677671759, -0.3447097778592457, -0.26029658897818037, -0.24086554857653256, -0.7901025595654412, 0.04040567304562923, -0.013057294125332687, -0.7014434859790738, -1.1529543283872048, -0.6964684657081871, -1.116584541175218, -0.9248805966464612, -0.885099032398658, -0.847680534124155, -1.0077408820279263, 0.5220342329576458, -2.0144374890866352],
[0.0740627384476959, -0.11494369957773345, -0.0973756703264467, 0.03290238393682734, 0.06310017642499426, 0.07538417501829472, 0.17963760310438803, -0.06587455373244172, -0.04007451618033869, -0.368452424455056, -0.3105058056080494, 0.11891873896435874, 0.06960330469737937, -0.09884733114506505, 0.08574316657560083, -0.4641812110861907, -0.3518014534015093, -0.1440769224684202, -0.19830053777283957, -0.2173075970944007, -0.15778477772967797, -0.00328423551395814, 0.08588522180804206, -0.1277303888257894, -0.4770952191903706, -0.3058619548656902, -0.30217550648885966, -0.1318059063323566, 0.37296024489013807, -0.062140524394229874, -0.38962120503095343, -0.3576834017721189, -0.22369412453281876, -0.5290695406927465, -0.5206114758114987, -0.5131059140385255, -0.25481266359962496, -1.5607221770663326, -1.3532793421075466, -1.570873583886981, -0.2764694155412281, -1.1272928873046009],
[-0.13751557260339495, -0.15554455336450665, -0.16968378725150782, -0.22574588797259112, 0.0785534239700845, 0.041061579983445, 0.024414182656632408, 0.09531361968821055, -0.35374289250815866, -0.4645669849893961, -0.5472264947030664, -0.007376995372728795, -0.14740814314920433, 0.05276679021221798, -0.22220007601583733, -0.650043666399781, -0.2452305212952527, -0.6781152265184279, -0.41371679963918967, -0.11019390087239396, -0.061047793724531084, -0.25185712634904184, -0.5549646618742102, -0.47624381996166754, -0.6540842543911395, 0.1486931362336801, -0.31699205843183886, -0.1471581277800727, -0.36721558178702796, -0.6478055377714285, -0.34211374505861397, -0.1568619722812089, 0.0025455962576312412, -0.44351436897396457, -0.33267524063103776, -0.5628144179205226, -0.30960792967616785, 0.14402818509911325, -1.1576445667655941, 0.015080860415062113, 0.0416532301298324, -0.8237341455309494],
[0.5855783429664121, -0.5043056040816816, 1.8214392566034086, 2.9407863839765898, 1.761327192114278, -3.253551081700134, 2.6410859897929253, 0.16771639280089504, -0.16401608320729374, 0.3797239635602182, -0.33549958210913233, -0.05937301609341589, -0.24335171957431972, -0.9287990605742749, 0.20447804175149642, -1.0071782370881834, -0.7257555413844444, -0.7606953143907331, 0.10197188964300792, -0.5863944698611575, -0.48299238469060923, -0.8185704535334588, -0.09897905292744401, -1.1026255447721736, -1.5612158215807685, 0.35861132104641336, -0.48528987575423066, -0.4114112428460436, -0.4785109089489899, 0.7338751117070691, -0.5466893340440999, -0.7341332682598551, -0.31824570065634444, 0.2332671120187724, -1.0755487399213732, -3.5561195780237163, -3.7944439774378242, -3.4055283134319216, -3.4324870687819287, -3.709534948069203, -2.808425085359653, -3.3641916591569263],
[-0.10500656581440186, 0.25568611045290135, 0.06686944220616778, 0.2717256677485801, -0.07790201238241017, -0.0052536823944670415, 0.1812433025916714, 0.030518536692271517, -0.00934305951373225, -0.15709820451948348, 0.011762615897822468, 0.03674859746680416, -0.2196639615890043, -0.03120255423299094, -0.15339831676835913, -0.10640000810184011, -0.2928411037785574, 0.10468839482681068, -0.22181286887093551, -0.2256595907235887, -0.07076642666520931, 0.0003180923576777297, -0.1526421297418499, -0.3011256585370617, -0.24012539121755463, -0.29227471653007303, -0.41044423982844463, -0.25204031283244144, -0.5438697480244727, -0.4350565611770716, -0.2574372911548643, -0.6076429743196973, -0.3437682078652119, -0.20074004346994523, -0.46921900137306705, -1.2932949357071575, -1.1832049959499527, -1.5124743536887912, -1.1911877227496293, -1.401284506989173, -1.1522712835080473, -1.6174169292739426],
[1.529198531398301, 0.40582264857680445, 3.605665356733346, -3.0552060956186295, -3.0761106239961635, -6.68399593930478, 8.411887685832554, -1.7398035127589409, 1.643140117472297, 1.4703548297115505, -1.0036717059462588, -1.365685242820113, -2.224689738373854, 3.0439334229495434, 1.647134549260529, -0.3281621170545374, 1.275469251572856, -1.7917955797589156, -0.5126482041831208, -3.048329836197423, 1.1615213474747015, -1.555390362066369, 0.7783940885746441, -0.6034249716048101, -2.8389441972928022, 1.4544291232398305, 2.7793984816990314, -0.709632596194744, -1.780294887899688, 0.9439581102566873, 0.3198335946732584, -2.198592031974473, 1.0797521902727283, 5.459027533301773, -3.0908089282690034, 1.4135379166146753, -2.578077653399447, -3.063340930849101, -0.554684581975893, 0.4820098744141777, -1.45313456616884, -2.7187668606271966],
[-0.264517035192644, 0.11379634779962515, 0.16821080743049088, 0.018887437035463087, -0.03305577640991127, -0.08044736230827837, -0.11718695342537999, -0.22228531097139337, -0.10706643042476714, 0.07184615267036638, 0.055488364518590134, -0.06310840195013515, 0.007111210336423927, -0.11274333772007249, -0.16121628075541386, -0.26353363511262046, -0.22029877207952073, -0.23826006665503047, -0.0629909163328318, -0.0899243918862659, 0.10286270727408722, -0.20406765530629967, 0.41570898609047313, -0.09300470018161838, -0.12695981875510745, -0.157858781591681, -0.25341848810438944, -0.03454245190995443, -0.7847725756631545, 0.3664716764588714, -0.5105436328330127, -0.6939454426842415, -0.2583448884812346, -0.24841580089756374, -0.1779079265584078, -0.7668120836347466, 0.11075210422420595, -1.070705566421258, -1.5637967811833688, -0.19522778573516095, -0.8672416757907045, -0.4588359783960938],
[0.445634376869198, -0.2722501579414244, -0.11389161462804868, -0.48583592942291504, -0.4977078308183644, -0.042318198287819186, 0.18724426455544663, 0.12081919414741364, -0.03867692755167608, -0.1492216846143374, -0.4853323852911005, -0.3981150390541051, -0.01222164223876941, 0.250876786918009, -0.3918954199865825, -0.23281480548503491, -0.5184601526842529, -0.49488061263003363, -0.9118307328264849, -0.4185492157488518, -0.8564225341461053, -0.23725544863294923, -0.57458826669032, 0.006869351730802673, -0.4096551651702497, -1.5299819295722845, -0.5904826508723083, 0.029572544479517217, -0.2828790888501326, -1.3356346486980069, -0.64704890928922, 0.2701285773225815, 0.17279219726145648, -1.3168148911878805, -0.5310731447111275, -2.3398789210036144, -0.9665636045915716, -1.4882172973319254, -1.8778716903962553, -1.1534959344552358, -0.9988567838652415, -1.1378913229602021],
[0.00924580206549492, 0.11342038950911325, -0.37251693998485835, 0.060447045355624264, -0.1660991834559394, 0.15717734859802787, 0.41690144871961216, -0.09269106049874491, 0.01596561593326644, -0.06865940280719088, -0.054240817451476055, -0.26668745029634505, -0.08002648386796749, 0.2399331024434432, -0.3561548046289615, 0.4611909773009611, -0.5416300539868092, -0.33972061600057096, -0.35981290730498916, 0.04980382740434095, -0.07184211960716477, -0.3440135760126125, -0.22099557705074924, -0.6550547054913372, -0.9114369792844443, -0.5354013191424043, -0.3798598002919481, -0.2896802971443894, -0.23225044453795185, 0.2388127080130108, 0.012572669747158292, -1.037990281636816, 0.05342849738453981, -0.7759033970866189, -0.7810460056959764, -1.9155275806581418, -1.4577223461112838, -2.3144175907595246, -1.4372380721676745, -2.2672141187874755, -1.855038454483971, -1.7264516066206508],
[0.029754400338719084, -0.1700354334259331, 0.34543274786976474, -0.20722676902629303, 0.059328100754769166, -0.14703331161587727, -0.14188042583934427, -0.14946766489554386, -0.0029519763905401764, 0.07682078371877861, -0.021447811678065247, -0.016662997893707336, -0.11651132331285279, -0.263188994972587, -0.16020889005409744, 0.02006164106508267, 0.04894622685228166, -0.3108806825581109, -0.14302865498443768, -0.20503110565696092, -0.14459316954569781, -0.12449987203072999, -0.416162647284534, -0.04132406534371852, -0.2189603741870681, -0.3703300892951006, -0.21935342162436663, -0.19468164867180882, -0.6842926412645487, -0.20509054114580227, -0.6560969278983401, -0.17484426306790699, -0.36358064612480995, -0.2553134124441744, -0.4655918338744343, -1.678028561196318, -1.3324129792663184, -0.8133441720702496, -2.465946778236043, -1.242153953935332, -1.1929506597669486, -1.046375653520704],
[-0.19342885337407448, -0.010068772364141524, 0.2947499436992018, 0.03701923373558123, -0.08453560529645372, -0.26712816415603574, 0.2540102548434861, -0.04110813259690234, -0.20546387861820992, 0.11620065727030744, -0.0317846645299984, -0.2814273487742857, -0.14224647304430627, 0.262560599477252, -0.100355703587657, -0.08199518771388105, 0.022147987299501085, -0.08828133312961434, -0.3821886872258041, -0.1085231538520983, 0.037529089769709735, -0.22929416192357663, -0.20572750195260228, -0.06038345083716478, -0.4514039679173564, -0.1895178818701631, -0.35340077337908216, -0.3138170904773038, -0.8054054105129423, -0.43351185110881657, -0.38170497323537295, -0.4441114562587969, -0.5463879881625845, -0.3987553976942729, -0.7461604619055513, -2.757558169883071, -2.4700340912283103, -2.958199355393034, -2.955086636998597, -2.9241859926467635, -2.718423290712943, -2.865100127771823],
[0.23009971827791337, 0.02032898439903954, 0.09779644401913261, -0.3494905471386057, -0.02361192195088238, 0.10842993175035091, 0.038704098155948156, -0.023413951737676825, -0.05580089427556753, -0.11290255495586991, -0.2651810181035564, -0.17808948909834846, 0.12633074823014037, -0.16791119674095129, -0.3449282520796506, -0.5888850442068811, -0.4267186247998319, -0.5674522145529892, -0.3957515759730428, -0.26152179058314107, -0.2510040765806066, -0.059007823042710204, -0.22702419696936266, -0.6935991871810182, -0.6384998805466467, -0.5508194746399417, -0.2937645120473569, -0.19076701293961865, -0.2738961499425683, -1.1135412161227298, -0.5784425526799039, 0.17894567416053536, -0.4174563181740497, -0.9702017078681187, -0.8070489264759526, -2.384710434058148, -1.1987679246755123, -1.7125432760038053, -1.9607489814442298, -1.615427073873279, -1.4868686193824265, -1.1606540579889015],
[0.056413972814120236, 0.3648461081178569, -0.5030945824570049, 1.1305497914118676, 1.2431469255996421, -0.4518465986255042, 0.24947065701113882, 0.37134501159479655, -0.1850659408811781, -0.5661124892750553, 0.5674800552128031, 0.7936217779921079, -1.255382237456545, 0.18249114326337842, -0.22317348553107533, 0.7244148661689419, -1.4910304275557362, 0.5204251772117114, 0.04927019232958273, 0.19671623757262072, 0.2999634056043527, -1.1303580776243047, -0.5177446876459374, -1.099585761110972, -1.1826179263194434, -0.05221529633297196, -0.4026220713743554, -0.7307016959663913, -0.6304455605528657, -0.10029718000604504, -1.5056304182807707, -0.5748333027518865, -1.1288999239073798, 0.7812030370813169, -0.19323430094873073, -1.8484489887299684, 0.04644387373938826, -2.3864910191847977, -0.11086675943472131, -3.090205158643712, -1.5789971533901381, -2.1336110980314436],
[-0.10285413217229943, -0.037203645552551776, -0.3835959711194885, -0.6642915651881202, 0.005701488825261555, 0.06335245995018521, 0.29651181010847233, -0.041061357545905036, 0.1004855630962036, -0.2967468928832292, -0.6317043013889413, -0.12373508224836072, -0.3101381997729651, 0.0064115757836438695, -0.11494008914109177, -0.10026342097781676, -0.08955016150776461, -0.4732925754342072, 0.05767707535573147, -0.5223617273851242, -0.3191650633370046, -0.2587914153532938, -0.3253798696138365, -0.20255812572836693, -0.4353363540729443, -0.2867036120550224, -0.5205741914610844, -0.3892228610891356, -0.773969963259031, -0.5736080398767489, -0.6626923908145252, -0.07866725675324743, -0.48336883424943056, -0.7851852253683519, -0.6248158433296203, -2.6004616236343994, -2.1276565728601216, -2.517808870675722, -2.8084404125816187, -2.5756000912907178, -2.2209672925427943, -2.6856664965623263],
[0.23990307756337992, 2.4264193716138083, 1.548938455769572, -0.8252869014729465, 0.9199179583782464, -0.9226116996927624, -0.9904003631835576, -0.21213562527278224, 0.9496580288201824, 0.5173940301919511, 0.05763484216967139, 0.723406286516224, -0.11046040109208793, -1.025769260782331, 0.5249424915033003, 0.4012027055724673, -0.2921591637885191, -0.3700892339628476, 0.3160659298673908, -0.587514433740958, -0.35414336432545185, 0.38972820653719153, -0.998669617483567, -0.8668376537600017, -0.6820761144384996, -0.11467137980955165, -0.15191783713971121, 0.2946883693760408, -0.7560530197161962, -1.3135496707134464, -0.8003795718920267, 0.7047660345136348, -0.9507398437337042, -0.2542195544061558, -0.7433740243623957, -2.723180817519794, -2.7793192947803513, -2.9956460288776285, -2.8927402649149414, -2.676102310072399, -2.5650956153115794, -2.927533355057525],
[0.1606588170541809, 0.17027961490903845, 0.04069845505246395, 0.17822394955293996, -0.07602018461019511, 0.07424394624332223, 0.03283636286544816, -0.09661702983203585, -0.10532426275403244, 0.03697366191100308, -0.08017146542204774, -0.09084319571197322, -0.08683842065324496, -0.008828050784963763, -0.06611395397150686, -0.06761755614977462, -0.13503706101909244, -0.048148954162059644, -0.20374973395019888, -0.250581436050768, 0.05959019149551829, -0.03512021159632194, 0.061130490429576974, -0.014507530767128475, -0.0744119024400115, -0.13577458678488202, -0.06733392321160786, -0.23057768303706747, -0.3534788322690986, 0.3782956126422951, -0.050632454594130044, -0.2894923715343934, -0.20535022857121862, 0.10840162576090037, -0.31919976680419837, 0.29654483304941404, 0.016207600357314052, -0.010160487076254257, -0.5257515913443473, -0.5803810013978241, -0.2214454161805347, -0.7340665945511048],
[-0.21280533274579516, -0.03808357566079444, -0.04973636448355387, -0.18396008949264542, 0.1008081629544835, -0.08846250037161452, -0.1070218117839449, -0.05253997370668869, 0.032250490378151975, -0.33779352606832236, -0.19529743630553054, 0.10972798224234924, 0.1095381216453023, -0.19456193103938235, -0.0741329375783775, -0.08450282110056195, -0.053903105306673735, -0.5171152672889511, -0.26702367524000975, -0.3028581161624801, -0.005352224990392568, 0.2178646359675815, 3.447834273974382e-05, -0.010474049073619018, -0.143399346148771, -0.22781711292645862, -0.05663324497325006, -0.13912706602228173, 0.25309766130013606, -0.247050493750234, -0.05359964679655633, 0.30040529291160906, -0.3662946457017347, -0.1727967324124859, -0.3412270044456925, 0.20942429488158967, -0.024081043993666115, -0.3371537501916552, -1.208694536333158, -1.034761293462853, 0.23457606297787414, -0.857207982735988],
[0.2165862433788767, 0.21282276204124276, 0.2601787052528286, 0.2747886647056306, -0.1422652520864052, 0.10562362005420171, -0.005676937949057545, 0.11090565032040135, -0.26427265251784476, 0.1160703785858753, 0.2521963951698063, -0.020980250488452836, -0.46973462692614465, 0.031858961308007805, -0.08146961325030308, -0.26438847923986664, -0.5514598675073916, -0.04979270557710285, -0.16825321638405338, 0.3314014995413804, 0.428936156660667, -0.4989421796619036, -1.0298099059069228, -0.5802225490456091, -0.602370451442146, 0.45935006570609316, -0.6340451142051062, 0.23102536528095108, -0.943606532876809, -0.24298385926513877, -0.43777851651514327, -0.6091186395931815, 0.7721178385721266, 0.9610723084677323, -0.08571582676780168, -1.360290721887323, -0.8811439475039028, -1.0676003718704294, -0.8506750287806253, 0.7906585495259387, -0.36980991701997973, -0.18540267833308763],
[0.1313560045896924, 0.07027235148401437, -0.0486836997008534, 0.05834670116778549, -0.005798612881036744, -0.08650717661664145, 0.19635237528128766, -0.04070301914507656, -0.3940433451275061, -0.3455961892095985, -0.16535710865523662, -0.07643086427550272, -0.11948699866329178, 0.1860437796163409, -0.12624051478533257, -0.3751454431585009, -0.5454754319175715, -0.3026119449255755, -0.4115169775412207, -0.17024639717456042, -0.045043427426856984, -0.07038420162208152, -0.550116428611353, -0.36452985283412676, -0.13874731107548968, -0.19999309284845523, -0.376866505428583, -0.36868861789013274, 0.08210219642702989, -0.7569304429230683, -0.20286463802437227, -0.2859846947191115, -0.4953612999281176, -0.8064276806974887, -0.7467626651792321, -1.4037298760876484, -0.8546951524113411, -1.2760503260262748, -1.1677691422196703, -1.526582939005969, -0.8976844320439563, -1.8118200850518051],
[-0.03264980857263796, -0.24411047901938154, -0.13925824328824055, -0.0697114294185283, -0.097940523160866, -0.2441390007997831, 0.0025457030390448735, -0.17511516807250435, -0.17496023659010793, -0.5704282974675737, -0.18359080798111674, -0.10542950179046734, -0.6874489056351681, 0.12853506652414026, -0.1801863190647031, -0.501836670525927, -0.773302796733918, 0.12528436303629734, -0.1266468424574046, 0.18146275157479178, -0.1545742319560368, -0.7181986607254512, -0.680648983470436, -0.9204215555770859, -0.5620958265925204, -0.09775583369563011, -0.9416605383209659, -0.7623984515147416, -0.5082689193091905, -0.582159341486649, -0.5744770305582573, -0.49502777284064126, -0.5946864593114309, 0.022388663019635865, -0.9954648458745603, -2.359600846343659, -1.5877222650402676, -1.9430095023352894, -1.2550715838065007, -2.3188877270789017, -1.0282604010805423, -2.050979043792995],
[-0.039236575308769894, 0.18957723886820763, 0.02759642694754166, -0.029667011682341593, -0.07009109494024408, -0.008104894758684594, -0.2314380080652614, 0.0004305226073059547, -0.1947300348584876, 0.013488138342547423, -0.20854160145485967, -0.2416045906378977, -0.16241620903528922, -0.12695944915958743, -0.3348319794020563, -0.26772219222765176, -0.3436861807991098, -0.23844850779200447, -0.26713841948959366, -0.09667924107644799, 0.1977266127075584, -0.20796541137957575, -0.2969794417236418, -0.2793361003485919, -0.3654290319638869, -0.45524224713490985, -0.10475672929597803, 0.01748932204530934, -0.6154949383658003, -0.6282207012052969, -0.5084971083744516, -0.23961190930694456, -0.6564579996786511, -0.10539313546177076, -0.573806054922359, -1.2701274660771253, -1.1936517534075228, -1.2894219863044303, -1.2779187812098707, -1.0632431370392033, -0.9318539122391496, -1.0053581988136218],
[0.14251617187669507, -0.07595403432266673, 0.1916499183943398, -0.10773367510262369, -0.1838183212257028, -0.030644698764994902, 0.07726306302393215, -0.17109037281027237, -0.17930875401607482, -0.2629448657582862, -0.23253138523828873, 0.1875101516640564, -0.2613033836952369, 0.26843836699991575, -0.1097196605810684, -0.26528112325342734, 0.3561768438840355, -0.48559158714997935, -0.17408467385136417, 0.24695179756020375, -0.2714361084807123, -0.4806560012109516, -0.6159016782432486, -0.130004142219109, -0.8226113174324337, -0.49652833783192923, -0.2953441916012368, -0.27255760877015944, -0.8953712940467506, -0.8077872166825762, -0.841866245928607, 0.3582644656801612, -0.4485948101521556, -0.5363765589524025, -0.6947823313434863, -2.4327869080258564, -1.866569631081814, -1.7800100938498213, -2.6205133201266784, -1.6314966265236184, -1.7981859187154614, -2.1280159602004196],
[-0.028155699642950456, -0.046222843160966066, -0.15928617970398382, -0.11974745328224179, -0.2413932226823347, 0.04795203173145688, 0.007899587297719713, -0.1787125151206303, -0.19279418741187948, -0.3205990787459229, -0.23372132087246966, -0.06832157596045366, 0.0773278512431438, 0.002473494252261412, -0.14338974492710727, -0.38628661973454964, -0.4375806160726275, -0.46869393304643503, -0.1305604121784238, 0.2984692969288411, -0.2610073541168542, -0.13964174212449032, -0.6140596004034689, -0.4009853682622051, -0.8224523135867975, -0.28239644859144275, -0.31667304680892117, -0.5365765563304544, -0.4036434277577818, -0.6230802444247222, -0.5279860896840651, -0.23775685902602703, -0.47482211620448805, -0.5506410713030405, -0.6405990247006164, -2.6115375560743264, -1.9515178933129886, -1.969839747779217, -2.517202934993054, -2.0704957274341536, -1.9482380897049012, -2.206377161744381],
[-0.0008220568820830499, 0.06411583731505585, -0.10597578289951358, -0.16022478733311069, -0.0605102864705644, -0.1371768984544295, -0.12056017345618658, -0.15429480709536753, 0.028460673608149478, -0.12557896237671332, -0.026304790188269107, -0.07195837707953534, -0.04720578658590123, -0.11759575114079045, -0.1712946322356544, 0.14547473334156671, -0.057977121382892655, 0.038946687636335395, -0.18975033901728958, -0.3810039900936936, -0.10046215727594195, 0.0036483215712362453, 0.0253120600467264, -0.13536329932213068, 0.038797184259670714, -0.08884951340664614, -0.170159895516495, -0.2790862499755871, 0.5353108198714952, 0.0035096917095910497, 0.5019288984456874, -0.0058025239074829095, -0.21869097102738025, -0.49293714107388764, -0.6261831526978695, 0.4929645210855477, -0.03749211430803726, 0.14296870201174175, -0.19405867655578468, -0.6614056820573654, -0.16581055717676657, -1.4017584031215793],
[-0.11043025808580229, -0.10000745310982931, -0.19423485710489183, -0.10967449011229574, -0.115217123463677, -0.051552942272136325, -0.04103023706610306, 0.005424594555932286, 0.030031772603221644, -0.10404291156252411, -0.26745093853379764, -0.16040928310254163, -0.08880354445963007, 0.001814326537902004, -0.15249303972414438, -0.09253137908429834, -0.18084726762602565, -0.38809255043924196, -0.25323638010541205, -0.29804141015156066, -0.1700374434173255, 0.03173430844887836, -0.13378109269802474, -0.14794336721712883, -0.3025791461889356, -0.2437997282375478, -0.17239166878621962, -0.1567076277033456, -0.2999955279770699, -0.24397388614198992, -0.5029471024928558, -0.2716112526150935, -0.4071764555407011, -0.42327342600526585, -0.48856218717618394, -0.6374010227199425, -0.6366866729530947, -0.8886509094793732, -1.5768924035230787, -0.9096663502853737, -0.5572020795156709, -0.9030673757129482],
[-0.01627448895116907, -0.02895934615321995, 0.01957733825906107, 0.07351424178820266, -0.19089815089913392, -0.19140359805719637, 0.000569781940872645, 0.10318822864907308, -0.03897224538274459, 0.10230166679512065, -0.21616231534034286, -0.14098944952504194, -0.1990881385395369, -0.0389272927725041, -0.20836029977978554, -0.22801209348072576, -0.21704703290869562, 0.03431408146985986, -0.1882384327649479, -0.12128627252585213, -0.24612896234237946, -0.3460225109096996, -0.32726010085116447, -0.44175226051975164, -0.4808530658633907, 7.357904496740275e-05, -0.3364207429422717, -0.20732590813193513, -0.4405481240911128, -0.3769845651011714, -0.4393851336368898, -0.518667522306666, -0.01702903907214164, -0.19987498658957983, -0.30052260263737246, -1.0520398576689851, -0.25718710637085473, -0.651227076237534, -0.8868657516280449, -0.30580821594737095, 0.035749817233983655, -0.6707207429732231],
[-1.9968355009386425, -3.533709692628826, 8.161611860985378, 0.6087044379853536, 2.8496611876166535, -5.442241012429457, 6.046707789040966, -0.07503746223233412, 1.0310305974593097, 3.1663198003083957, 0.23773429756645292, 1.1149107552005133, -0.957096016413552, 4.025951905206899, -1.2789809842000843, -0.7841617792420469, 0.36904107874264364, 1.0417428360786039, 0.098705874140309, 0.5097725649246062, 1.088818244996391, 3.46909217845317, -0.12707371864485975, -0.6440726665020159, -0.015304419178939798, -0.25036364596753974, 1.183822479080591, -1.6210921382656656, 1.2861908490536673, 2.469321559243234, -1.8827401688483936, -2.373561291892128, 0.6206798099636667, 1.5254562147085058, -2.8143416798098637, -1.6347279608641172, -1.0753463423599532, -3.922298620500358, -4.66273694239757, -0.678717667112646, -0.005982691550151079, -3.7791034659404725],
[0.22567927078159108, -0.1407774709766994, 0.28762856887818755, 0.031004989916789554, -0.022362900155990555, 0.21845895039587965, 0.05215776871810938, 0.04356806355314594, -0.01322745642815503, 0.0981765861895421, -0.09799381176351611, 0.12469249870875902, 0.06323956055369359, -0.003945429999256353, -0.021981411681077628, -0.04459273696561661, 0.28254001901848064, 0.05731660405317287, 0.027895648251577008, -0.23712145290891054, -0.16975355479044224, -0.2571647469446516, -0.49336463069797115, -0.22261897140980044, -0.2451643749745694, -0.292668527946383, -0.21552501560047635, -0.2490337656788613, -0.7721332751832517, -0.40581584295175516, -0.4245821692435614, -0.2918371649578043, -0.34560958852887286, -0.5792710972220917, -0.8995457295963625, -2.516321042733089, -2.1382959580161196, -2.2577407681520576, -2.6994338095486916, -2.5360123528114675, -2.2038848771427206, -2.3765673848676365],
[0.0379352774945801, 0.16254966137524868, -0.15890274442862912, 0.08173424873488468, 0.009201850999513188, -0.12511833071633185, 0.24681298796404413, 0.06357620394356979, 0.13569515881175848, -0.07753265813873969, -0.2094856120230813, 0.060425622711976254, -0.37115508946772724, 0.03550212373674317, 0.1438600092352435, -0.1592395600859209, -0.129707578678343, -0.3492357570226575, -0.16094781387830767, -0.16515331705496958, -0.155534031665564, -0.31500386647163586, -0.4394735141831861, -0.3766922831591542, -0.5830809836093239, -0.008261060593507848, -0.836109967990897, -0.26856094437713457, -0.7639351821214575, -0.20077102698340366, -0.6178874210829223, -0.5883491537605716, -0.28456270311026866, -0.4706188180879476, -0.17454949814081422, -1.5824084562684084, -0.893820829367298, -1.385998929248721, -1.7100080881123818, -1.3208877169599622, -0.9585321493799926, -1.232601026950783],
[-0.043865003928779255, 0.023689277218310906, 0.6845265701229465, 0.3033162474191728, -0.2704499277806669, -0.013621434184479097, 0.09242799023327285, -0.08217527567254475, 0.12534195017425145, 0.5564162288143815, 0.03170950336117217, -0.5399346027852959, -0.18241432820790202, 0.2601604486812928, 0.2247899001015157, 0.42641651084909643, -0.023824459360239872, -0.09342670466323674, -0.36007766020271437, -0.3680335859177969, -0.017443610734151734, -0.5589130014941496, 0.0016948701360856386, 0.17122619970055036, -0.3726498216667631, -0.4886167379998708, -0.7927665607397361, -0.17013292521613235, -1.1951461626580409, 0.662461057586693, -0.43776014678441416, -0.344082317153971, -0.7472905081008009, -0.1860541394175765, -0.49655602516283154, -1.3277999943615093, -0.49701934605609777, -1.0525689144268817, -1.6808462945651177, -1.0912344603880226, -1.2789722850174883, -1.0476465753769213],
[0.48976098181734273, 0.04625013583927609, -0.15479622450233435, 0.07850481314927861, 0.31423990851556916, -0.23915273884774269, 0.015393901111070668, 0.36067975263213115, -0.006270849619291838, -0.0997581576223526, -0.11196585762375588, 0.19678501898221246, -0.2616837440225373, 0.002525981692858913, 0.07398384471522926, -0.21325451055044445, -0.09002589577884086, -0.02830669169076331, -0.22225408595520899, -0.11970919969526225, -0.21201481232800695, -0.3865700611707051, -0.5022986372294379, -0.09977524808065184, -0.16310782936171062, -0.27319639842049664, -0.1871718329833737, -0.1837434565235184, -0.7115166535212509, -0.4692855389284634, -0.32893321119719604, -0.2509009080793897, -0.2528795998610407, 0.024998335250725194, -0.22356412964896613, -1.8011456458351718, -1.2622359765474722, -0.4446099152079358, -1.8971946904534507, -0.6674439722877101, -0.8510052857569732, -0.8565914646504503],
[5.61303752551176, 6.3689408387361555, 6.035224378507758, -1.535022187494333, -1.2968972975666129, -5.056766570322343, -3.814981524801555, 1.287905702334813, 5.418508974327776, 3.235428285390886, -0.3655574630476915, -0.2706601843362518, -1.6139500656092816, -1.9321761047394195, 0.5735019384712561, 2.2387671731119223, 2.7832942426108542, 3.388902898673908, -3.5163340669963916, -1.8329581930496701, -1.256395175898974, -1.2980214164423234, -1.7455598149319693, -1.6615361891858407, 0.9000339531156257, -0.9481472733510121, -1.3342818223584756, 2.609354292682355, -3.7488799124338468, -3.1691982793123255, -0.5299477308388577, 2.0799596152715805, 1.7969953504264045, 0.49298497452405654, 1.776277144347177, -2.6823820799260942, -2.3179983259013075, -4.463066607242357, -2.1352946798641295, -1.0584612863245102, -0.8152522615707674, -2.3337599965700218],
[0.3783752309273066, -0.26925905805261485, -0.07135438692135515, 0.13987123892054687, -0.16134612378228552, 0.1186694223547217, 0.5134359949199555, 0.1975405825037905, -0.47106713784020965, -0.09234601997666748, -0.43557482138959586, -0.10504174554788602, -0.01333859138461942, 0.328676001951972, 0.15060549730411946, -0.6854648523822456, 0.1750764172054134, -0.563442553566653, -0.23750482379434795, 0.042678143651646384, 0.02199358751163952, -0.28041240376537185, -1.0083771346817176, 0.36537186004791844, -0.8090282153645532, -0.4579987310627341, -0.18333521170465697, -0.2560886432419324, -0.4291730182356728, -0.5545742165530101, 0.0005331236847189833, -0.5584510638143384, -0.7675390398969373, -1.118928878268153, -0.5916997408125261, -2.325930573885732, -1.8587046732709653, -1.708495254387342, -2.6648034659968753, -2.2625304784741473, -1.9864665475809056, -2.230143644144486],
[-0.29612397725828593, -0.1861806852920056, -0.1197558453596678, -0.030883556460419445, 0.18953054910154887, -0.04652240096418896, 0.1988740874496356, -0.36069413366732966, -0.12455889368569872, -0.2560496288671991, -0.19681728365486545, -0.16470350731181488, 0.09743558031417933, 0.11436873886268631, -0.3273046911174764, -0.11489034871752324, -0.17856097169298013, -0.042612764694886306, -0.15848976265132766, -0.2803058037112985, 0.05088518101076629, -0.3557449518642716, -0.09754560957014481, -0.31528392743842687, -0.3896756494899538, -0.2804699572708243, -0.26823866745840824, -0.32368122141986627, -0.5330333165937774, -0.27134877171204197, -0.39944371783287924, -0.2728681598791119, -0.5648236353912158, -0.6495950149750528, -0.7923945736326903, -1.7056085824384628, -1.7802535405832751, -2.0923964037092944, -2.364030298400854, -2.281081730863959, -2.1001991657419987, -2.5110634385509805],
[-0.06069787207924032, -0.08472744498392093, -0.22410571940755566, -0.4821181987070924, -0.12918384539285604, -0.05685354964269523, 0.048127194827466496, -0.09823324780241541, 0.7268030801319374, -0.7736356968607624, -0.9991530571600751, -0.34208638727849955, 1.2212716582706902, -0.20232328367858324, -0.44785005959748875, -0.20523237975942435, 1.997196610310262, -3.0835828895146093, -0.06116798340157759, -0.27823388699055984, -0.6450222987083285, 0.6644589606098866, 0.9942635531496065, 0.06190831021219803, 0.6406007213768102, -0.6766907433560465, 1.0718858974731496, 0.4067236757707723, -0.11530235353254019, -2.0150211679045094, -0.2769156912748192, -0.9156340205210023, -1.5799181253884163, -1.1434392549573775, -0.9840100899749746, -3.2056524531034416, 1.5143443257798654, -1.7848349388215146, 1.3016393290421926, -0.2219160036652563, -1.7983839683528533, -2.658160829209266],
[-0.1539167750385967, -0.0042046436418503325, 0.10236521537657868, 0.3231174946525954, 0.12154745876111964, -0.09429962974461042, 0.13545776983773242, -0.25630541740737744, -0.14256788510617763, -0.158702008168659, -0.18367428470018976, 0.022924909371963675, -0.2892512009115382, 0.10416048273737817, -0.06313740412716498, -0.15282045255615553, 0.09845316429388609, -0.04693568721444418, 0.13979101830030036, -0.17219766457497415, -0.053374856117775864, -0.26909781371882474, -0.3240987175012985, 0.13618075722224854, -0.031164702972287715, 0.01484623094268373, -0.517798820017343, -0.39202944097810244, -0.9850448700119505, -0.4161030081004984, -0.23549591062953099, -0.12769436769239823, -0.4673161222117944, -0.37249884019853463, -0.8750383504163677, -3.9038223421662193, -3.5951956025579004, -3.928091786135147, -3.9872884421183814, -4.020467153476689, -3.810434100667416, -4.00318190168552],
[0.06729343222311081, 0.11513833915204191, -0.008450508840517375, -0.0978036254638027, -0.07858015062217942, -0.033208258412431406, 0.13177553070887627, -0.0554891174642299, -0.22046168872969002, -0.031146094735823127, -0.077460026551796, -0.2289261996020659, 0.04749400724023165, -0.12510727946033368, -0.15099371799399514, -0.12876205029513094, 0.06561031290114289, -0.13670821335339461, -0.19565963825465935, -0.13285415809803994, 0.05342575335089909, -0.40113161557276095, -0.19914743267975688, -0.09594615563703605, -0.30024433103049875, -0.17589069996805629, -0.31489818079461046, -0.27910562297333186, -0.74897993555597, -0.5710058622858747, -0.5717876662849191, -0.3680721931490382, -0.3778626199269761, -0.5100716274457117, -0.8121457802317182, -2.9285585521908537, -2.5616599668343834, -2.8121499843280136, -3.018257516885536, -2.941996518420903, -2.6579620411100393, -2.8539019834114128],
[0.00982184468976981, -0.04307887775284377, -0.035788637645491804, 0.0997397660627905, 0.08632796931627802, -0.1409641436163226, -0.1542090463484863, -0.1850168654514162, -0.23268875052977117, -0.09761153942555667, -0.019498033046974592, -0.19090715065261651, -0.04191383440499459, -0.06721827974507817, -0.05880681175332931, -0.3076461263548984, -0.08464704475985733, -0.32426126861654003, -0.18034505888122518, -0.2631701128019467, -0.015621392389623732, -0.2446778591354121, 0.13055063345358908, -0.12381454576056554, -0.3703416698965216, -0.37150901252917307, -0.3661411378177179, -0.14550978318948943, -0.8982739980792556, 0.39085540291269444, -0.5652946358508792, -0.45565305760029984, -0.2686303685721389, -0.19691318357162127, -0.4690599968332504, -0.5336256849793508, -0.4490954062040707, -1.3287660215502886, -1.6418347072954713, -1.141816702854721, -1.084996632746226, -1.2108480675223279],
[-0.16990562903300116, 0.3420488428637751, -0.19478348275601992, 0.01732876325093218, 0.02158818030984397, -0.21612948727739506, 0.023904979798716437, -0.006585943946309299, -0.08319942897288934, -0.11754228062182752, -0.16408519555485357, -0.24454844521354788, -0.2790908222411697, -0.06393046107890217, -0.11755607725877479, -0.05299231343952646, -0.19185622898694643, -0.18925598567330595, -0.2600681340147717, -0.00012317948212962697, 0.01706248158189726, -0.14415400645386386, -0.3593004836928248, -0.30180390779173555, -0.3246799503184774, -0.051054500466614354, -0.2967166223547328, -0.18031247563100816, -0.5704683420472954, -0.451725230763166, -0.1473101913863441, -0.297070444612192, -0.4488432955177526, -0.365293785234347, -0.6780199235203764, -1.454339427863356, -1.5639342556349705, -1.911150603552012, -1.5856649644370093, -1.8465713481651675, -1.7339008050391314, -1.9017646818964722],
[0.11799288219001006, -0.02143250011009339, 0.33882965094652145, 0.14875211614266656, 0.2626836029847292, -0.2375617531967097, -0.08453388265269918, -0.03395939499907211, 0.11599935150737622, 0.10907755872424105, -0.18098764574049175, -0.0669510961806019, -0.3250081628331085, -0.3394700087095587, -0.13032145838120968, -0.12798193991626156, -0.018400864023640544, -0.3678923267868688, -0.047708785719558326, -0.29963917860000616, -0.11429557257411431, -0.20855267685914483, -0.5558569265561736, -0.12012150647598703, -0.3693962082048156, 0.014244550396673042, -0.4921382305886859, 0.04538390237948121, -0.49550115831459196, -0.6317714672226306, -0.3693164575979431, -0.4556363258429322, -0.018871702300599837, -0.2451847408709061, 0.0655210232970796, -1.0212282079339403, -0.3700326671038479, -0.57101936935227, -1.6412798811532898, -0.02113208903848644, -0.47264639753385623, -0.13890775495681487],
[0.1639979110634622, -0.12452698153074303, -0.026578357677615974, -0.08396926947573773, 0.08441170052327325, 0.2680820312055214, 0.19966258890789876, -0.19496652139166573, -0.4077703008374287, -0.4745903880852054, -0.15737735290941265, -0.1431770396273357, 0.047911902070819475, 0.04906007559894011, -0.13699563553165067, 0.10597349610860861, -0.693195110169974, -0.04863417357789573, -0.14840030327306195, -0.2605357049845291, 0.1575785096456405, -0.19544560002899947, -0.825311219676292, -0.30411835954140815, -0.592371031312891, -0.43015516315454566, -0.52039819474235, -0.47993609284872274, -0.8835866055155718, -0.2837658694764519, -0.5571676503842027, -0.011304066300580975, -0.4937339105981466, -0.5769248867611969, -0.6235767276599493, -2.358861918102407, -2.010410545105273, -2.3921891216666142, -2.7770983077598252, -2.4160954717718526, -2.2994090593326018, -2.627945422412207],
[-0.03345211424485114, -0.122364237143227, -0.11437543679058254, -0.4453911697726687, -0.8544431200365333, 0.2844731464906452, 0.20257708886243814, -0.3658529974386898, -0.08890207107719704, -0.040085142692553144, -0.5543185119090317, -0.7170189388349175, 0.2462968312392155, -0.17176902979547126, -0.21683519293500764, 0.08219429440886457, -0.4378994781799418, -0.6692312234058726, -0.8554069207640725, -0.23017698442877688, -0.5342266589532788, -0.3847537244262463, -0.29773527706044367, 0.31360840347425517, -0.3538032349105264, -1.8149811405901148, -0.49965498937680086, -0.46109625861552184, -0.5722265115981016, -0.711330377253975, -0.7672307066476043, 1.0708102851979402, 0.8813171030191471, -1.1239087995847334, -1.1550273450853672, -2.0740106297974377, -1.5612234972921815, -0.8302552763799622, -2.0949036552873332, -1.135145015679117, -0.9214151521480289, -0.20937422172768194],
[0.047387945942363535, -0.33786970279978706, 0.28110220823147114, -0.19465241369413236, 0.06990741661489623, 0.3149255020258597, -0.026465671755911236, 0.1412705851269641, -0.08379272704783125, 0.13863854003178097, -0.48653308724853483, 0.11514231891178223, 0.08686818316215085, -0.25873171018968555, -0.10217679357383024, 0.003560207707716949, 0.1943599598624037, -0.15348301384383933, -0.1705094485959868, -0.3534752216553484, -0.253285593201649, -0.03916061093075455, -0.11075685714896355, -0.09169500599109427, -0.13000515407903565, -0.336681265380594, -0.22094599829122372, -0.36410288550765446, -0.8269712533215643, -0.3794508023427539, -0.4701423164009916, -0.25000680456074653, -0.44391975533259487, -0.5927771507769478, -0.8479814519472737, -3.2235914364953953, -2.867817931006699, -3.224516717224806, -3.4927167755422888, -3.394118184264419, -3.029814855761544, -3.2769229822699284],
[-0.1840985504547821, -0.02308613779551145, 0.06407901556799409, -0.220558990626051, 0.07996266500500203, 0.011969723773741624, 0.09853816951279838, -0.07744493517276256, 0.11526132960229275, 0.006152514522274071, -0.18554089653904782, 0.1297356480376889, 0.04004565529882413, -0.055573002416578074, 0.004501252516644127, -0.06669234525541308, 0.0534521217308479, -0.2990697566552207, -0.0658747580596389, -0.3155471551742702, -0.2305227386768981, -0.14307775484199398, -0.36469145498273015, -0.3753213198176552, -0.44426149614018995, -0.2209672703956179, -0.3242792085176794, -0.3514039304267578, -0.6510874600590015, -0.5104348188995939, -0.4886905226534172, -0.0996610826348628, -0.4495673542381914, -0.5610410166700002, -0.7821674080695432, -2.2427711631260987, -1.841668836374144, -1.6831598426450896, -2.515215090679988, -2.1444409764193506, -1.7398653163334086, -2.1937337630634244],
[-0.27968801667059, 0.19612213633713993, 0.1776431028410961, -0.11081284739990833, -0.16725543955736016, -0.3546856774469066, 0.1519271062112225, -0.20603168481681594, -0.14361971853095204, -0.12494781601498597, 0.053379872283694355, -0.08703978819670036, -0.3129392997991305, -0.026015459631333913, -0.029132430255833078, -0.18268525683833647, -0.17414801202862154, -0.21596593206955778, 0.011929679018982583, -0.2513716391514795, -0.07185773496440484, -0.2579184234650753, -0.1578379850052008, -0.2863278880270475, -0.32185372756436276, -0.1437457190010898, -0.526598741164655, -0.11124794907590696, -0.7515477367514867, -0.3678515209829566, -0.4757014718799221, -0.36393637074114094, -0.4823087871297608, -0.21430928303641375, -0.681117927797647, -1.4521442977476564, -1.5743206123557674, -1.8687741747894604, -1.921731696486286, -1.6312751334111848, -1.7152882813501906, -2.031906054177053],
[-0.1553778145011602, -0.029168963570283875, -0.20989401974788666, -0.17580432988019282, 0.0027153649949931947, -0.14001984064396342, -0.030825954036894542, -0.20496116613121948, 0.11683945505341327, -0.11040945518485647, -0.23031934386476163, -0.0606537765103087, -0.03950587900334649, -0.16664143652164057, -0.12236324650958678, -0.06759895803441313, -0.0564318672157533, -0.18942738284656518, -0.04580549734212948, -0.1633506654919999, -0.1842944353407218, -0.16075150606095828, -0.22846601998489122, -0.3111024339471583, -0.44367861090225974, -0.2161697942436293, -0.3823099816249046, -0.3369502375288805, -0.7369819360588994, -0.600787338679167, -0.29260881521941196, -0.37556935500396077, -0.4947805599345857, -0.3490595445279996, -0.6367333902458888, -2.0712119881546247, -1.7976401102186184, -2.177436901428867, -2.4048009555302445, -2.0101170380316185, -1.9119593928754526, -2.1343955052069066],
[-0.11824695850636521, -0.2669695337013634, -0.20953645853920608, -0.21163305202047686, 0.07433307026393793, -0.11145768203003828, -0.061252505231101456, -0.24541303212491983, -0.2146567692363606, -0.0039970218687855955, -0.23718622884032478, -0.014479622791459363, -0.13863958471275062, -0.016629306399698653, -0.27807816493909443, -0.29545141387178125, 0.20403574520385478, -0.6357644583771975, -0.0530370796398199, -0.4085202549195427, -0.24400201891299336, -0.2839423778368861, -0.45750275958570025, -0.05997812560080107, -0.24388306777407195, -0.17004568868710454, -0.28058266172971524, -0.30070218854695946, -0.6625082163992949, -0.19873848429315025, -0.28474673810731704, -0.41243426383902143, -0.4090387209514134, -0.5849942507919195, -0.5619692819893439, -1.3977793476743339, -1.3576841916800044, -0.9021275519834502, -2.3000499766640057, -1.46850768549648, -1.5598532191426966, -2.1467556786305146],
[-0.09782819802241137, -0.18636893944950383, -0.1727317140639487, 0.011769194953162367, -0.10198252307676141, -0.04648774513104721, 0.030727384238963693, -0.2871368280865965, -0.1375018212711921, -0.2881931852929202, -0.05431976196666219, -0.10860644625405917, -0.2373347183498279, -0.15227650234921195, -0.07198977505381697, -0.1551837820254847, -0.23197185557960426, -0.5388891529002217, -0.2530742570883448, -0.15107504084183196, -0.07653239809052963, -0.35912028881361635, -0.14193774416257368, -0.5772665425614439, -0.39951121316548577, -0.271259510135614, -0.4738396784667757, -0.056159667448253944, -0.6210112625224155, -0.3744837093376726, -0.4969581533066345, -0.3047726109689725, -0.24852329142705631, -0.4078021950649696, -0.7767557006409932, -1.9422525060873475, -1.702923057868728, -2.0071467783125403, -1.9345412331999103, -1.898443505259494, -1.633676045681382, -1.6952284151465882],
[-0.08178734178475229, 0.18825046325396358, 0.06387651159638895, 0.16661766353018181, 0.006030787596513659, -0.1966385424140326, -0.17039345025463748, -0.0404198082622109, -0.2159037369875289, -0.08777536069738223, -0.12704808606191426, -0.1981076925235787, -0.10467923684887076, -0.1547028586626386, -0.3688770612587648, 0.05301496032822059, -0.5519560450698853, -0.21674126433571844, -0.20283206960837308, -0.165506562232382, -0.08821802518624725, -0.03424108125763572, -0.3280510482733229, -0.44472925953016634, -0.23195574165027052, -0.12558120688444238, -0.8591525484084495, -0.24241752576376835, -0.3246930120796039, -0.7468161554105668, -0.5681955530367611, -0.5839286771393757, -0.03572954254695573, -0.20652733237263574, -0.7146424652953225, -1.9206488018838532, -1.341135502697229, -1.7038285237591992, -1.5699972751211217, -1.1226161028658825, -1.2770481970766638, -1.2871049570988211],
[0.6453544582889031, -0.43962982246148286, 0.6435346155785334, 0.5228693317984273, -0.5165364816201687, 0.13015620652146237, 0.3645725667928911, 0.20217368001345984, -0.3896372341777633, -0.11062816654778353, 0.0307022105398425, -0.4889734561368893, -0.38503948201621974, 0.5598196533449578, 0.06432506000763577, -0.4827472905511749, -0.1419769237520182, 0.10797820985213309, -0.3593596988616699, -0.3854517529273324, -0.11489725294829146, -0.3158725674666267, -0.37098556777355224, -0.15226134733420016, -0.22766657334752152, -0.7347621304610346, -0.714936910037842, 0.010015385217492612, -0.83137269245602, -0.5091958790923659, -0.810480394142149, 0.14787728204867576, -0.29916267977372346, -0.79158346405738, -0.7281301150617975, -3.704433440340733, -3.1095326906494525, -3.560720838938593, -3.7903476114312236, -3.444148870390647, -3.4016986489489813, -3.5973472945986846],
[0.03274310305223328, -0.061261428820748476, -0.27208061474534745, -0.11030894151973832, 0.08792638553812719, 0.023238671705338977, 0.0015692032813048574, 0.04448419134784019, -0.18592858521465183, -0.40594032227311605, -0.3093564013575132, 0.10912990461589357, -0.3740271280245674, -0.28684811438726965, -0.1319026778719648, -0.151196294308735, -0.5150626004304744, 0.9247758098511667, 0.3231009541098614, 0.028300798729322937, -0.2482884098060235, -0.34809703992644675, -1.1192407012384096, -0.9056418493921521, -0.12797670022188537, 0.08500687988606259, -1.0508893897914453, -0.6081276745707824, -0.8592241088303237, 0.0391522104391649, -0.8679390104764488, -0.7810281550722379, -0.31322079473487685, 0.21046114748966954, -0.5455995784388306, -0.24147174661412657, 0.19189840468204675, -1.3829463991143083, -0.037035358769476084, -1.6566656924602339, 0.7400943783040026, -1.2267009393938788],
[0.003155587848635416, 0.15324330200690747, -0.08107540295379007, 0.2503214959241925, -0.09187253015177832, 0.10767229215881623, 0.052095465173589726, 0.09064226470181999, 0.08292064022166189, -0.32322291787876145, -0.11768547368917956, 0.06062281927951771, -0.22029408288126617, 0.0631319388532409, 0.0011292827734840508, -0.16771165250102166, -0.4100773532424242, 0.041146780063729266, -0.2898729283817148, 0.05829799394813021, 0.03788852389985852, -0.08591050873097662, -0.3120427157059925, -0.43475733719950743, -0.17397005862500817, -0.11409181583327588, -0.38548208151664676, -0.4626605544653033, -0.683026401794756, -0.5120326996996308, -0.14576050343533212, -0.49508142021568763, -0.5382252202859072, -0.5001531304263215, -0.7787324432502837, -2.7664208642796493, -2.5023094986195726, -2.813724864797378, -2.384722392806542, -3.056664883971798, -2.694489142599226, -2.8848673315506526],
[-0.1922245489422293, -0.026852926643106425, -0.017637763611574635, 0.1914562196232123, -0.03091949814581632, 0.05314829019686425, 0.06475466409035284, -0.08114980023446815, -0.01255315990366548, -0.0789588451211839, 0.15569543079504233, -0.25143094051667325, -0.22227727821703055, 0.06103279889909174, -0.4511146379392687, 0.394615373927282, -0.32354016620968284, -0.10927063219524116, -0.4609158201855771, -0.3026828148595931, -0.1311666758450586, -0.4104921273337309, -0.45202900274771834, -0.8883033177958816, -0.5816658359492768, -0.46959402398983097, -0.750173179066612, -0.49268531443558905, -0.6552401713421391, 0.0787707597478928, -0.6841817512250303, -0.48477356174524616, -0.40030728168316165, -0.23904250299440152, -0.7781553906373172, -1.0099706930517893, -0.7454566735893585, -1.4039923521906152, -0.9391972494193943, -1.2694351432709319, -0.9040847087237279, -1.6761180882888285],
[0.1175599212475801, -0.03121164423744745, -0.1713494267804806, 0.11377501659234189, -0.35229731037316453, 0.292807701304696, -0.0753603162400802, -0.11458268490135033, -0.4400364675898396, -0.2574159933350901, -0.03900494918892141, -0.19158983999770005, 0.26301068028012414, -0.20649521213460797, -0.34298412778964943, -0.05916865904582804, -0.31576169666887827, -0.1722290110728509, -0.23993513912666303, -0.16081890192290024, -0.2507754945862924, -0.5452995131275707, -0.6235451003287378, -0.5324827330408742, -0.5614649738361275, -0.6528666664648638, -0.3651578048000707, -0.4233163449883528, -0.8676978418125804, -0.5643501580335559, -0.6935131270546363, -0.44377219674030316, -0.7692680255041773, -0.6134451729039194, -0.9430554324885778, -3.7667332845728083, -3.2310141234044294, -3.637646745381676, -3.770193071888545, -3.649947879430177, -3.3785775480968234, -3.6288708378682286],
[0.061846729829299604, -0.020243546850899366, -0.44224800742174475, 0.23470566770605497, -0.247744473326063, -0.02908208773788919, 0.36297315547203257, 0.00862284230344043, -0.11971880646755902, -0.35669466650179976, -0.10907528749634272, -0.2450581737321911, -0.12339617391540451, 0.2876808979940205, -0.16709785589522555, -0.033579487387843256, -0.2810542102684428, 0.1192205044593306, -0.27630410993519244, -0.07526729473033301, -0.2187071236960068, -0.2009996254302993, -0.3656190361946158, -0.2826048582299674, -0.110901099015404, -0.4311839379589255, -0.4371163758182144, -0.10218599141908513, -0.47976888932655504, -0.6611958853771992, 0.010137113517408532, -0.42161411912560237, -0.39497710726176555, -0.4728090898154796, -0.7332027304374554, -1.969331795235106, -1.6874396487391654, -1.9778468318512412, -1.4921133123175079, -2.301056376192703, -2.110583128505772, -2.0131150994308715],
[-0.47527894718925284, 0.06366771435042524, 0.1864963037910935, -0.22882339893292597, -0.06318929463198703, 0.26070787366500264, -0.2842948115878682, -0.3770241034324828, 0.025679443983239382, 0.0675884461802801, -0.024759263851301096, -0.21379943756479958, 0.33343456112992825, -0.293015787230964, -0.1380235093337016, 0.14341102300239447, -0.05868604318952187, -0.2717460065392681, 0.021965091488046644, 0.06624890638619034, -0.014717538783582302, -0.11684014168968881, 0.14831373972705533, -0.12543498318322474, -0.20073172214449658, -0.4215940637763204, -0.131473181781629, -0.32561635990664617, -0.5743460396804132, 0.06124222226375634, -0.4895220056168166, -0.22784539368352558, -0.46479554075038554, -0.7356534722080665, -0.6678046261325455, -0.8812739744771995, -1.001619276516235, -1.395273392610919, -2.3243897744025563, -1.8158871643978072, -1.2893836076996759, -1.239406879098076],
[-0.2528518405483855, -0.14857205630897835, -0.08281617513285172, -0.3502926289766647, -0.1501987969384308, 0.008840594663763216, -0.059579072776013854, -0.10723726363453824, -0.07114104195328225, -0.004181198786835399, -0.30015044738460445, -0.0798823398783381, 0.0979864929250663, -0.10828440331525059, 0.27425574401402847, 0.01012032394009197, -0.14751896373896944, -0.18691572050343142, -0.01775395535561585, -0.4055370762850024, 0.07922030119814374, -0.05699572900349496, 0.11897537694881057, -0.3700477308587561, -0.4212998278751892, -0.131226729358594, -0.03051983651165624, -0.3288903721861315, -0.6040703048598962, -0.16764202880277188, -0.74982306566887, 0.1402226682535819, -0.480334860806381, -0.6337634230644683, -0.3250977413467637, -0.5072627123879238, -0.3054846572312087, -0.979342070391669, -2.5074710077249316, -1.0476234773407433, -0.3581060445824069, -1.0891357269701087],
[-1.0971582838917968, 2.9488669415933084, 1.7509938650190953, -0.7660793069767701, -0.33981206818695436, -0.0318467078930586, -0.9888632397510043, -0.29667503749408464, 1.0947032429877264, 1.2184384672669857, -0.4611209782047395, -0.26696807233987435, 0.40693722722523235, -0.7990299762491282, -0.4809424448361797, 0.4578668524863575, 0.033720903326610284, -0.5578048902741112, -0.3429479556023107, -0.6344832760376198, -0.7533034810401029, 0.21813437331356372, -0.7106069537111693, -0.371194308336262, -0.1000656694757345, -0.5846576705632914, -0.24990623170088627, -0.11038306878277941, -0.6352974436981281, -1.6784436017169715, -0.7930198046925647, 0.22794292312088693, -0.7408159935728816, -1.1333797518202, -0.6890513529245834, -2.4247562773316087, -2.905898474113934, -2.9589309989654264, -3.1771713079674493, -2.4493123720415744, -2.42381719896396, -2.715167459588979],
[3.7649276082474357, -1.6506638554556001, -0.17433063699415263, 1.4147841998097557, -1.0257284796571458, 1.138552816386895, -0.6698419013289572, 1.5109070686516024, 0.0041219649864762484, -0.5885802225569865, -0.09726508232507444, -0.07646720745111507, 0.5000983953432605, -0.5997553079008737, -0.7172893078778241, 0.0592905188244658, -0.2978560542300753, -0.38697623521972263, -0.8201484037885227, 0.011120664933202563, -0.5873359062390695, -0.43502604129616507, -0.906245267216259, -0.2357839778215522, -0.946845003167076, 0.28266741664628897, -0.6596123570468546, -0.6263530647282489, -1.5259377931962694, -0.7271999654873925, -0.23358378235302169, -1.1110567907088338, -0.6908639727841452, -1.2340787051598088, -0.6290411237812434, -3.48724375173418, -2.911350435271411, -3.0572537726957894, -2.56027092969341, -3.1009041826357766, -2.67780837531625, -3.285305355365087],
[-0.008953647681650839, -0.005806764380813857, 0.0384788687200972, -0.22729684445123022, -0.0074123289380132774, -0.06034256699138538, 0.039518689818241734, -0.10537389749716394, -0.04251789766346256, -0.1810843035550149, -0.09169527401981718, -0.11441804767812173, -0.04677517878359427, -0.12250008803873444, -0.09877684581353796, -0.34181354647369677, -0.19676260282105265, -0.4068040319307872, -0.291966561241155, -0.43284690914522433, -0.1729692035980535, -0.09685929533788534, -0.0009235068943940609, -0.20101198284066468, -0.4739294529173017, -0.15944729113483017, 0.10393851583362254, -0.17183849670691592, -0.6583656446701952, -0.4224742483590228, -0.6524009262417199, -0.4545177928331309, -0.46329657837749333, -0.5404396652391568, -0.5293925796586155, -1.5254015762645075, -1.4932921802448607, -1.1858848242545865, -2.413035501081317, -1.5274976548252173, -1.348435973924578, -1.9393234060704208],
[-0.015987834950120773, 0.12638970420137774, -0.0615630308372475, 0.24847576033498409, 0.2167313256864595, 0.003048623283964219, -0.03989991186101945, 0.11308897368236981, -0.08498863798805226, -0.04999490460673423, -0.1449287258837997, 0.14259337549911255, -0.17307176446172645, -0.0732323984165833, 0.03573841210482732, -0.2278813293352349, 0.024410308654775358, -0.034673802659395785, 0.08804267351672156, 0.25517689616334055, 0.2348811089530273, -0.3942729141973747, -0.36302875668709483, -0.3452668657692724, -0.3478951299537202, -0.2729679192612595, -0.1644769489265675, -0.28955843310695095, -0.5782944012027389, -0.2895467935749884, -0.7947674808063592, -0.5147285463309123, -0.2552575463757773, -0.1861293697733156, -0.5036687279257216, -1.6533328358754777, -1.1137613413654397, -1.4300447455398702, -1.5935715987732668, -1.2445216801358212, -1.112560309899289, -1.1854430873956692],
[0.1773815906447209, 0.040302193115756106, -0.13804164824193552, 0.02946468733411025, -0.1835543478603739, -0.055462606168744356, 0.08286056773222275, 0.012242829069312755, 0.02339686997371652, -0.42761171165233425, -0.1270015610053137, -0.2160189661156781, -0.1127637713656404, -0.12120052679149544, -0.22501757702221076, -0.34816882998326726, -0.7191031605087229, 0.0609959692085146, -0.6859227009124083, -0.41602292316494044, -0.25621785590561924, -0.09788351298967045, -0.33976561446430137, -0.2466611724449831, -0.4524446354577727, -0.4067494294823991, -0.8853429546131819, -0.4215153913644345, 0.3863524177242776, -1.614040979756546, 0.803074930099336, 0.828444786888381, -0.9580739182103192, -1.1680368011880364, -0.7886138164861358, -1.5772782211416265, -0.7300958037095695, -1.5993481461428658, -1.7072065694730945, -2.191755836242702, 1.190065461150228, -2.335019149750172],
[0.01689001825002413, -0.16814712487067784, -0.4148182032023112, -0.30261656268099574, 0.15862096126857922, -0.0233683667295358, 0.4605329791432187, 0.1630845806545483, -0.30569249314770564, -0.5902462496068216, -0.2291872717087781, 0.4786896486479534, -0.28085082085566093, 0.11904736625337625, -0.07051967321097036, -0.41333045479512304, -0.2336635204106702, -0.09707574360827645, 0.4463900537512713, -0.06579723492947509, 0.09637243627441604, -0.11104264533323752, -0.45486925731935857, -0.5428361969883502, -0.36195318167325086, 0.09152053711104148, -0.5029117474308135, -0.3045481977381174, 0.011162248460661196, -0.513821527388818, -0.16279380918287026, -0.2998330271575705, 0.006653711478801781, -0.18111402247128378, -0.7519969119696265, -0.8322928042495586, -0.28439713011078926, -1.1114664967662267, -0.8512019783335298, -0.6401612746293036, -1.2350354597856708, -2.121591586319547],
[-0.028062374419221058, 0.10546550771596029, 0.1390901234545524, -0.05490545701382787, -0.10588454577298902, -0.2272769945898361, 0.05295747624107872, 0.07991807186999488, -0.12056163002312613, 0.04547566533423808, -0.032100839938119345, -0.08545136107256115, -0.20083405355322523, 0.04240169612225022, -0.23514527612650823, -0.17975716176812367, 0.009271471839331908, 0.0033908925699899107, -0.46781501884807025, -0.2749894848927034, -0.24047655795078396, -0.37027713888361297, -0.60217378232923, -0.42425535585213636, -0.46482621825292375, -0.7611902720710237, -0.8576229714050472, -0.5967629688106834, -0.795689259329464, -0.42457281642792194, -0.7427318324001067, -0.3702760820327091, -0.6736602033799661, -0.39844425485669566, -0.6783547782581428, -2.4405404812717006, -1.735280679393489, -2.236513908752501, -2.4968452976435067, -2.3890078535254258, -2.08924136153646, -2.400720743636935],
[-0.1987767055886416, -0.24941062391363394, -0.1820713115826418, -0.08394082142697493, -0.49515500026486586, 0.5593708412981288, 0.399408437156922, -0.06880681956377087, -0.23245173994962315, -0.46597298320166664, -0.2630792277231765, -0.05179389079290582, 0.4135850586900022, 0.43886426457837485, -0.31003648799401895, 0.15180943489016807, -0.3395210282575643, -0.7681133422211728, 0.03648866825789475, -0.041789550202708775, 0.15272110038598874, 0.051171109518103546, -0.35613406074635867, -0.025264889160017873, -0.3220844386804225, -0.09886673816490117, -0.40655961489147996, -0.1628702403932865, 0.39702728559981626, -0.596657874647591, 0.05913974983871785, -0.4682193317239991, -0.2515624719538694, -1.5072479074135903, -0.8424933416627672, -0.1309035178755594, -0.8476817287740963, -0.8178732308699217, -2.084214996067312, -1.7395417112230214, -1.3114503543497438, -1.615259148315187],
[0.3417358333621668, -0.43793982956655303, -0.3017609749146018, 0.11530974732329509, 0.3591070231179148, 0.05536133114561405, -0.25931202331804415, 0.2677727301928181, -0.3051675002054451, -0.276680842972824, -0.0036817564044082877, 0.2920906754725564, -0.3428319876749976, -0.15997860848974169, 0.12103588903399268, -0.238271097994857, -0.0548191886483274, -0.0034621474437391635, 0.14170321208996492, -0.010551899561024983, -0.24503243810064504, -0.3969790184264035, -0.5131523530587672, -0.30889363548928295, -0.2236112055357704, -0.3443322919069559, -0.3729778162049714, -0.37592479735023543, -0.20516749735731404, -0.3493516845888718, -0.34361423977979166, -0.6539846827457946, -0.8101241099699515, -0.20267152399407143, -0.7468489979027398, -2.453360791791181, -1.7445382904245264, -1.9981860361275612, -1.7658496158312382, -2.088262894024284, -1.7364351821141615, -2.0823432696235065],
[0.6051920626890576, -1.0480948733048034, 0.2017246169515252, -1.4005803189018926, -0.28122348113929624, 0.3007225226115654, -0.01779011506907949, -1.8305682853156475, -2.3980047328546372, 0.023770547247936927, -2.8595092058955616, -0.7816481546565454, 0.17550011021697626, -0.35389772533520436, 0.4203126639526983, -3.4219999258837355, 6.160886385736878, -0.12621651859982772, -0.1881209124251342, -3.0655364200929758, 0.22772828398442116, -0.8697498242688586, -2.719718868760815, 1.530443615828802, -1.3147812692412393, -0.5174001348495191, -3.6078322331696833, -0.907229778097852, -0.6371577449449487, 0.02671874063210968, 2.0178028817717517, -0.7180139419171426, 0.43005249837869375, 0.4448977925161614, -1.45432083780088, -0.22438624567315377, -1.4480480112136203, -0.2535230208445185, -2.9284388292691363, -0.5017940621110384, -2.195693766468227, -1.734194937493279],
[3.202225860918579, 5.83555974375786, -3.605141107129782, 0.4476945354854715, -0.18345746187720666, -3.0151389623755356, 2.411044825525223, 0.15536063636116043, 2.0923020887992907, -1.5135225202862639, -0.7131733073522276, -0.39259214655476643, -2.5889835826058247, 1.3487065368613587, 0.48985356552408016, 1.432528456318557, -1.2878871542485715, -1.1876776743776774, -0.982838433902817, -0.5757279760673578, 0.15288880147159695, -1.222164805926103, -1.9763308706601748, 0.14923821855660963, -0.2106459068338361, -0.4667796487304669, -1.6371673036848509, -0.8540449292494329, -2.5263529579102175, -2.611957301465281, 1.8661743864527522, -1.1214961485727655, -0.030236533563014645, 0.5983649455221273, 0.5436590918518149, -1.5178162777524034, -4.510723905128795, -3.249391890840144, 0.19009884193288334, -3.2172310668487993, -1.895869143053303, -1.3902479736588649],
[-0.4854169497424214, -0.2907885859600759, 0.008442540281278773, -0.6218143095434596, -0.15837930595113361, 0.6708384202361418, -0.1414032348216039, -0.26159859370020816, -0.0783903493919502, -0.3770165701788319, -0.28772341670798923, 0.1744963429013197, 0.3897380489089695, -0.1830012133844296, -0.11419172847045049, -0.171378020642974, -0.23041034524790097, -0.4398376345127165, 0.07145815981097764, -0.09743331148484395, -0.19969401676442208, -0.4031337298854122, 0.11526570179301465, -0.43705680989877066, -0.2703971024737473, -0.35435711409168175, -0.2794019101473995, -0.5131625465803508, -0.6810752905080619, -0.12349897879240873, -0.7444016980881325, -0.236979373232363, -0.3397695476310235, -1.0253346901496485, -0.8457952016615066, -2.8153758723051077, -2.4272717158552766, -3.1994808796207623, -2.9240656088691472, -3.1113454918801153, -2.697604203907204, -3.0903289783952155],
[0.3211319923039462, -1.5978995410549277, 0.1150662394163089, 0.2415329450278854, -1.1582109794545987, 15.878094877160072, 1.47511121178312, -0.10612415349464373, -1.1288556900084188, 0.9212540210757676, -1.1572283639396865, 0.06703989604314976, 4.212164589463876, 0.9798645640414403, -0.021292177057128707, -1.2210210465449078, -0.7621366406332647, -1.2960904580214536, 1.4575576922479199, 1.2403832756356123, 1.0246706677456432, -1.3236751872128139, 1.1359244892199378, -0.8508335033738694, -2.1721017945697985, 0.8277604879208157, -0.3412946116297508, -0.07690793140623918, 0.6460898398224382, 2.6997929835016423, 1.3960784109671418, 0.30018757897051346, 0.5533003652154701, -3.0990640982236863, -1.5037759894832543, -0.18489635722547065, -0.45103981523115777, -1.901700537589423, 1.4228341251067762, -3.5960134403641986, -6.735162384627211, -0.5291310878979721],
[-0.11420844225181563, -0.4791406182799428, 0.799878985763397, 0.4669233985850197, 0.1864024361074052, -0.17630896409266159, 0.04683609737737884, -0.3919046591550765, -1.1224185230223134, 0.7719385428820917, -0.33298188235244713, -0.4923566345320361, -0.6869488052017809, -0.012663455178493526, -0.18871657280231915, -0.13877461236800018, -0.330583785653178, -0.045246891179150046, 0.10983520522682323, -0.6872186364322236, -0.3040621000270894, -0.9695097027493593, -0.4818262460414394, 0.7989339008267723, -0.24126607630769456, -0.7150861126335537, -0.23963690983942193, 0.16676094856018628, -1.4713714958861113, 0.42125574577826597, -0.08453854829246202, -0.2512553193792717, -1.366106396572933, 0.11454532157030725, -0.07457138103383017, -2.532283862289545, -1.283606239082872, -0.664908861866516, -1.3890947711404507, -1.3961406809475134, -0.2164565547398185, -1.254562834795605],
[0.20812040524006728, -0.3661253369239581, -0.9296763683962688, -0.13806016673007038, -0.28397919256681947, 0.5605156310815402, 0.390857300644061, 0.16535070038636437, -0.38638095783207366, -0.9904390191454349, -0.4322129090698699, -0.08969997035118446, 0.4493586247918769, 0.13586052876162144, -0.3908602280964605, -0.12863643512508321, -0.38061655442492814, -0.0914376089702946, -0.21238565166808793, 0.1488707242105485, -0.006227455203697551, -0.46106750325319795, -0.4905079549531228, -0.4758192615274964, -0.01728626138071551, -0.2911539562510066, -0.23101715199609674, -0.0984827663345044, 0.23175713321004712, -0.8920398053039728, 0.06681289407089638, -0.2512229072289864, -0.5059067699952217, -1.1751893340089716, -0.7439421981907555, -0.8886127038476366, -0.8976973014373117, -0.9762110214651168, -0.854759422514894, -1.6758061014347063, -1.521072524142408, -1.5768305205488962],
[0.06933843010686072, 0.08848619465368943, -0.027782017584054432, -0.26529236365085385, -0.19592380970774162, -0.2982964554675507, -0.1479061460611785, -0.17624923553183855, 0.18596133308434282, -0.12341564424266102, -0.016489647728285973, -0.23201393189472372, -0.02840315014021277, -0.0664572556933218, -0.25711162326565606, -0.12427706872901133, -0.10113359900971611, -0.394701753123266, -0.4647747707276236, -0.20591260643262305, -0.2744621425866012, 0.08815298648113788, -0.2975464951713699, -0.21393474260913264, -0.1704353850543879, -0.41202984950825083, -0.3618861578585821, -0.25762010713257966, -0.2148963012073618, -0.624691272539259, -0.39999423261660805, -0.09333883731335949, -0.22650178578408015, -0.6306426620651869, -0.5051603398303126, -1.0583449559126001, -0.5093569575074549, -0.5895660338491545, -0.8025722759677362, -0.6280964947478422, 0.0754648804347071, -0.9486603699070174],
[0.40006629433031077, -0.27838461053051333, -0.13716454123627042, -0.2403346591210388, -0.2785057391890633, -0.16547072824413597, 0.11997831486585224, -0.09531005076636467, -0.26332050165877163, 0.07711049347674766, -0.31546667090425656, -0.21905495164976588, -0.41820751007121987, 0.0007635629478651402, -0.09632992644023801, -0.46333509366737247, -0.029421685270935333, -0.4941423067380119, -0.07351317083619609, -0.5006262973460629, -0.030843435689543452, -0.41145133948265145, -0.48909768923177416, -0.23934125642404877, -0.4346267005431363, -0.11666950479867494, -0.6526267420912154, -0.20717094948498943, -0.5640224574032052, -0.5725151738343576, -0.38926765589739404, -0.4823505462402314, -0.24063042415659427, -0.6730602152267667, -0.3106673508510959, -1.6965552923688476, -0.8614250190490403, -0.661667375043997, -1.929417246862353, -0.514049488544689, -1.0630207328554284, -1.4825196581780282],
[0.08015528192109303, -0.0814759917270937, 0.028135477904299374, 0.2283569634342394, 0.1022750331806769, 0.04293476787234212, 0.013917948349052535, -0.07305581345793277, -0.052804390198586494, -0.323875215763721, -0.2704669328944854, -0.06146507645452246, -0.10722387930460679, 0.023722379051571443, -0.14215957527794848, -0.4473714356079949, -0.161290031809396, -0.30989268798783265, -0.20109810582944473, 0.028318985315921915, -0.20172246131934796, -0.218063585159297, -0.5531260367726074, -0.41151344585492977, -0.5486090027250775, 0.06747678272598437, -0.27094504327593744, -0.3263234815163878, -0.3897639767915259, -0.8190183405360532, -0.4970701138598662, -0.47748598347812493, -0.3023333014418654, -0.4784795928666103, -0.7424626866815498, -2.1731840466749404, -1.6282021602067251, -1.9883859431163224, -1.9230565966006528, -2.022910551464184, -1.7300401961557204, -1.9205768721842706],
[-0.020090136845584902, 0.012074403863646893, 0.062315125426697586, -0.1235405383971911, -0.22994302180057613, -0.16171333173840824, -0.14428043734636617, -0.20134432540428887, 0.08479613017803043, -0.07841298395141312, -0.24163249690733618, -0.04961845190421214, 0.03304749967976671, 0.022084327139819206, -0.011340524705528427, -0.09330924487444847, -0.0016475089980669331, -0.3710632499089209, -0.21626967925826585, -0.14649805809326324, -0.1506131571249697, -0.3553286170830298, -0.1503316018864559, 0.09118747868089887, -0.33612601288436006, -0.19473317281108274, -0.15503002540233157, -0.21990238544360097, -0.5103005864069974, -0.22101923473936394, -0.11646625111148246, -0.16246130455846802, -0.26365224361944034, -0.4511820147719811, -0.38890508122167583, -0.7772312663865373, -0.7021557718192463, 0.5343492316623515, -2.01513181690389, -0.5010671811801216, -0.4085021613899691, -0.6512226027879291],
[-0.04725994342488882, 0.036385499196725714, 0.01997286365170255, -0.4985383998821648, 0.02323438327930261, 0.1427187327366944, -0.04805272421628892, -0.450301821725267, 0.044129425923536164, -0.2451441644342898, -0.49936414810088453, -0.2027273040377944, -0.07840471632919427, 0.22561523936024513, 0.08148395320136997, 0.2895914964091133, -0.4197873679438127, -0.876732223597011, -0.20314307932249917, -0.10513537147588267, -0.007047493304466888, -0.4124209748753382, -0.027359198281357387, 0.28273638669074136, -0.7191553201826071, -0.62818620453419, -0.5131883396912811, -0.1294537234476594, -0.579465451315338, 0.1800840877357522, -0.6704142855935477, -0.5251136942421767, -0.5449564126223072, -0.7118197121639246, -0.7242132518352392, -0.7215970974442936, -0.6548512323842278, -1.03601952325424, -2.122892284610352, -1.687574054174749, -0.9768955016861399, -1.3751042883898739],
[0.10836716760741104, 0.10134902605587917, 0.18764882084457046, -0.5697747970413729, 0.13804209786915153, -0.05003318664666909, -0.04890548057326867, -0.1260548990283774, 0.09906185857426221, -0.019461861117684887, -0.30765490549819535, 0.1692226114477184, -0.1445940226761024, -0.15934581191835065, -0.0801710676670721, -0.08014408997330448, 0.16890787528338888, -0.23281204881021916, 0.07400253493285358, 0.040566192927615735, -0.04736282913196623, -0.22940018066396917, -0.33606230268372755, -0.25186935884801714, -0.45016904578901457, -0.008231963500450783, -0.14826058483877338, -0.37084929652437504, -0.7630014209801043, -0.6907443313226849, -0.4353448950871504, -0.1359525662339651, -0.6003099161810846, -0.4568487329030626, -0.6451133977367275, -2.4068088457085066, -2.062325754509267, -2.3077266160329306, -2.554001083037291, -2.117558489872852, -2.1062140615443616, -2.520483642104378],
[0.19615672096447667, -0.14497530087379493, 0.0035600711482192612, -0.442005461492972, 0.04698639742217385, 0.24292147464215522, -0.022331295489419353, -0.23548501358491872, -0.33931743839225037, -0.061430701824919234, -0.47696055023942724, 0.08079243422256402, 0.01869166643961031, -0.24256089631239652, -0.1893240699401945, 0.0007481064766160558, -0.35745383354139515, -0.48147586369269035, -0.16502073739789463, -0.07869164025618008, -0.39506543242295833, -0.3885199895271729, -0.37936186395183724, 0.059173624758963345, -0.7318073024217736, -0.33736840399599777, -0.0602743147204111, -0.18112509620962564, -0.10427822023492105, -0.3641867748745078, -0.17932634169481657, 0.13289737385880362, -0.3895731478148392, -0.737805612037719, -0.4236417449851737, -0.24638347951224032, -0.02418701313588499, 0.3234963500660435, -2.175968388276442, -1.3015086785757044, 0.09914832236297896, -0.7782818045467647],
[-0.11087852790487489, 0.04406683652319311, 0.13123035534165098, -0.04410780294632531, -0.12338552237820227, -0.11084106249854489, -0.07587763855983601, -0.15084478651929134, -0.13556967932949748, 0.013474932603818936, -0.19177230060651287, -0.09568778979262234, -0.22091816786991955, -0.020238404352230246, -0.034649846163409, 0.0180069302935321, -0.19270258757503161, -0.33946461806560624, -0.2887060616549752, -0.14594683589665539, -0.25634706105038246, -0.39255697004331114, -0.054474696624344746, -0.1490847231601107, -0.3036167761866691, -0.3111787840808683, 0.0680814630375453, 0.12705762781616925, -0.6406814627401025, -0.13045167874296415, -0.336476229082135, 0.023547094419207305, -0.4341582117344413, -0.23789297973347717, 0.09648060312619797, -0.40367731689642883, -0.745707301510531, -0.13380905340023716, -0.8808926175674943, -0.24521717377753596, -0.4613043964926156, -0.2554600376629233],
[-0.13596624545293856, -0.37240228441598805, 0.2171145359731681, -0.14070170818059427, 0.40661751774770044, 0.12930353862283464, -0.02875535693690299, 0.10840469837896737, 0.034274168420905866, -0.8767097558459613, -0.2957335666036015, 0.32797907193940395, 0.9140181643306547, -0.05555360109259599, -0.18948816787098507, -0.3916666578902981, 0.7676999347805379, -0.7172696077294596, 0.13321910076105772, 0.5138216658948468, -0.5635693853756863, -0.6241886373007259, 0.45629991388563396, -0.6373456139487543, 0.6648993408251894, 0.05940190675302537, 1.7272972812652514, -0.5727665914085658, -0.9263170629891693, -0.5728873222799683, -0.36941316929480916, 0.3079981863089511, -0.0035269651088409995, -0.18545412466266364, -1.5066425956718181, -1.5661935007570638, 0.08756810648117411, -1.150314843054875, 0.8805535485190481, -0.179584610535093, 1.1394605797098316, -2.3070735175563675],
[-0.021371566421534368, 0.013905755161235099, -0.17978704871274262, -0.24875625867904666, 0.06481553751130008, 0.028109200793122638, 0.0173533266600947, 0.01517254592885474, 0.010425738690264821, -0.3197505358509196, -0.14515694328814818, -0.006687229656639013, -0.09543302575195163, 0.07625721421293244, -0.23807910687918074, -0.2939076028329405, -0.14338631862583237, -0.18374946177685292, -0.10116477500340153, -0.33177449853936886, 0.0058453282088485985, -0.24276051529007311, -0.29317958008296924, -0.34462954003126706, -0.5939056228075401, -0.22726871669909443, -0.28944095857472324, -0.29958997215309574, -0.6206242464311303, -0.5584839310788768, -0.37972965718138674, -0.23851560694735305, -0.39576000870275324, -0.35445083543452016, -0.6727678326725164, -1.3448755376817203, -1.2480406191879054, -1.0240014137592843, -1.959096898676881, -1.3085235112039522, -1.2558140894174497, -1.727973116821733],
[0.039013969765345304, -0.7377570412488775, 0.6915190801421096, 0.04603743172106388, 0.015907460824867676, 0.2466048015811749, 0.8815073005763269, 0.3074728690634304, -0.9120378090884426, -0.033945025448113605, -0.5066972437292002, 0.21261123891867253, -0.2997104450699783, 0.3170380173310166, 0.34211165104765473, -1.826900001282943, -0.24340789781603162, 0.19677565872182903, -0.10461864076434538, -0.41171441623366783, -0.43167792363325996, -0.0017369231980309797, -2.8740611540098446, -0.21971180598752832, 0.6860371533541046, -0.2924743495506958, -1.1266248817256113, -0.5582822739253724, -0.8280290851343621, 0.8102552749405705, 0.15345056600022103, -1.2669956611843964, 0.4365119770664661, 1.3756571456902054, -0.9154770007831795, -1.567259248228504, 0.3469557796734618, -2.062410692460657, -1.0170869574866888, -0.4650803298889439, -1.946025664047504, -3.170133559222733],
[0.015190189024458404, 0.14587561093188894, -0.32105572535255, 0.18147020772644776, -0.08802524551685062, 0.12728926334006252, 0.008294056041835206, -0.010602691840883176, 0.23783372487771928, -0.34567506998210557, 0.018298382149544466, -0.20259973716291302, -0.03098054711681051, 0.052865938575601, -0.13705646107359412, 0.005859586093063076, -0.2793392289799668, 0.0023987148707919066, -0.19857897823672613, -0.11404119855363028, 0.03894004059019545, -0.23188475702924422, -0.21972932350301613, -0.3618839239730095, -0.3931339572034674, -0.3458071961642965, -0.31664285105507783, -0.32386425449941003, -0.4824213692104676, -0.4929449849172771, -0.1450007982901551, -0.424526900846695, -0.39098711656785573, -0.4918486747392897, -0.7146735695451321, -1.297531270380233, -1.4570228092291742, -1.756490366631406, -1.4891270240091743, -1.8046761698468776, -1.6929776955847347, -1.7901583613589416],
[-0.45721047189019043, -0.3472355070754812, -0.31599237247415773, -0.10001940946257666, -0.022007051788545312, 0.4701364083758199, 0.0317006529856945, 0.0263402142096009, -0.03927553559894454, -0.30221943142423013, -0.04485960555422678, 0.05211491886626002, 0.27008966049183164, -0.014120562577674254, -0.08197254990692146, 0.005932778678133667, -0.37669095222646737, -0.07994087424944077, 0.0876627792021754, -0.19286777480619094, 0.017608606391069394, -0.03713959508426063, -0.17638201477897805, 0.1627093755506006, -0.5517560801960424, -0.3457892516855633, -0.3721266385638272, -0.2967387820513027, -0.10825118603664055, -0.1977915328149155, -0.005827141140310346, -0.4156246054255255, -0.04392183159127367, -0.8440051632381373, -0.6801832392090461, -0.9376332021566314, -1.048035071924909, -1.6202093156472988, -1.019582545587699, -2.1382664062836034, -1.389051710591124, -1.8331720589964369],
[-0.0914385390801705, -0.1817972782148749, -0.17596616483162697, -0.22184138370467946, 0.08161202660980535, 0.026660437396352002, -0.0869109192417683, -0.13277723264475386, -0.16164597800827954, -0.009062501636178806, -0.04901585202282189, 0.06142262995513774, -0.14828623963344537, 0.0026511850808604923, -0.010249885510096021, -0.04261334624649008, -0.10537843671478214, -0.23544992377205554, -0.22382332877584601, -0.1707664249831556, -0.28802175295248617, -0.17981030669198664, -0.22611814161807667, -0.2787043111059058, -0.4393725846324645, -0.39729315817076694, -0.3401674788080098, -0.03076321520546085, -0.5163688793677349, -0.44706007786524327, -0.4252773065289593, -0.24232076832440771, -0.3456615567752032, -0.569953499900011, -0.305698958589096, -0.7377122652496775, -0.740799306161057, -0.7640143775613415, -1.6058062972980365, -1.1198513030733854, -1.0275975870678442, -1.2531647281918104],
[0.4808016179785692, -0.427794261234402, 0.34053863009423174, 0.4548754651754978, -0.15254796602269358, 0.0900133819860566, 0.020483491837805037, 0.1084128760982413, -0.36991064485510944, -0.014156198408098232, 0.11788523072368397, 0.09410010394976155, -0.0063276607888351705, -0.1282228342247098, 0.006292264381128863, -0.3553889440983901, -0.03408599125178845, -0.011772983751511692, -0.28423692702499365, -0.18794761121874662, -0.2906468723744229, -0.44128358748503255, -0.8386444732570085, -0.011884476563619292, -0.25180837928043115, -0.29136555193362756, -0.08558857297120218, -0.2570982575831674, -0.724714595124787, -0.16119367428640355, -0.48498087290281355, -0.5858224673661846, -0.22525770829347896, -0.3449225792495626, -0.5573902415422347, -1.7547404422350412, -1.349152954812004, -0.7654847174476826, -1.9289073304278606, -1.2970805963784005, -0.9180015487822345, -0.8327209067010436],
[0.5695206942808111, -0.04280607277357573, 0.02995132256434074, -0.4375854841872729, -0.43841103241621276, -0.2604701849330998, 0.0774067849288501, 0.244535587870221, 0.2236894496811294, -1.3213817806651444, -0.5261976899244905, 0.14516477060986147, 0.24775418217349682, 0.13672949099028187, -0.10352710865429741, -0.1466656814969944, 0.4872929114526348, -1.8775816882149647, -0.3051209590500854, -0.6096879113376878, -0.7191200707942071, -0.73662503862978, -0.731983042378223, 2.2376739237748393, -0.3920747843303132, -1.9883324388412948, -0.9339425348355144, -0.20503116882258293, 0.22266336651292293, -1.8820204425549367, 0.47536230468189256, 2.245275842641132, 0.323418424125792, -1.860099594550744, -0.9435880156601585, -2.191434769980248, 1.5131374515501026, -3.501418307709936, -3.2117455385670373, -2.1116325574058754, 0.6817625347449986, -1.3430080714551622],
[-0.3857153862051739, -0.7128244652385827, -1.8353100692525872, -0.6410419209441576, -0.6904276959321465, -0.6856878653846564, 0.9070572492403807, 0.19996836734313445, -0.8081966928640013, -2.364194049711614, -0.8843313907624031, 0.8100302268004045, -0.37687787860494415, 0.33084910596769557, -0.7744863564579398, 0.1094176549012135, -0.5217025494735057, 1.6223280947948684, -1.9293250410436766, 0.8166803453459046, -0.3998658148065243, -0.1231543554409715, 1.4768369230448148, 0.5490177270401724, -0.9842941048068721, 1.0151932513709554, -1.1374229698543572, -1.5243342668145987, -0.6905532877386664, 1.7139545368951, 1.4807750740168848, -0.5921247340913293, 0.2685775512224261, -2.4280271872251307, -2.168521874431425, -1.2030349368194926, 0.9604289560576008, 0.018034490566892082, -4.283854081641493, 0.8955658843692359, -1.0464736690791692, -2.1661096733209284],
[1.9906543543126667, 0.829543498878448, -2.5746496516847692, -1.1642160328429245, 1.6686290934268577, 2.2102925308991397, -2.150308671289353, 0.8388955807369748, 0.09704933460494031, -1.0853110035241715, -1.2225237570199483, 0.282518048193499, 2.064708172629873, -1.0421471775360522, 0.7564266841644761, -0.5085354653031593, -1.201448293339939, -1.2550903173298689, 0.2054592645278352, 3.0631801452208136, -0.6013399545787306, -0.6541528754737548, -0.7201417448417383, -0.7223436219408115, 0.9189762140789728, -1.4272617755134633, 2.4861692369607074, 1.1458510452162702, -1.5285174062224605, -2.037185994730791, -0.04830463421190717, 0.2450912524941907, -2.4944532656892235, 1.3143211122907394, 0.24611002041970562, -1.8820888762031058, -1.5001976493325806, -1.2345888832608647, -2.793425554029928, -2.971273726221923, 0.5173400993451344, -1.8531626604824307],
[-0.08345843092505667, 0.171455024695179, -0.08138549584085927, -0.1545783636600558, -0.22282755526669537, -0.20124276193311194, -0.02662017550456014, 0.05838570770712476, -0.04275009973076641, 0.3773401498551246, -0.14366319927356502, 0.011383685780078978, -0.27716095165450616, 0.23390263801996386, -0.20618630788613448, -0.6354105450425422, -0.703541366661796, -0.1728471504513779, -0.3332852266092178, -0.30240556698509274, 0.20207507976244696, -0.48334690205207925, 0.301693394288246, -0.3241810914806961, -0.9746841817323441, -0.6016332929231286, -0.560014395021335, -0.1857346632606397, -0.61066998836741, 0.10820201361123138, -0.20317846868521058, -0.5518787469959557, -0.9255147029937233, -0.010097095966721988, -0.23425625765372338, -1.3046778918191715, 0.7734459810431745, 0.6624450550620716, -0.4727798157681209, -0.7674104525057723, -0.797612323480374, -0.30755109398969044],
[-0.959288885008894, -0.2492189223235499, -1.1173112379762724, 0.7497935564891238, 16.071364161255122, -1.1279196393449933, -1.593901028216088, -0.17302821718670403, 0.04131336839255339, -0.6512854067542432, -0.4063865006124096, 2.5827505810695097, -1.056880457772265, 0.7868014782476298, -1.114209192191814, -0.13284526592896748, -1.5560379220307685, -0.4518653276100855, 0.8696410665217256, 1.0026952802220415, 1.1208231897943726, 0.7740841746981771, 0.7562056705142486, 1.291823207402551, 0.5604251344729685, -0.3210373459714327, 1.0400548126558362, 2.165688096181959, 1.3530248566369687, 0.2526976625613932, 2.3866806427420064, 4.240269216918325, -3.3267732931212306, 1.19092460366137, -0.036373027317585474, -0.48559673544213716, -0.2421237791644053, -0.5748238175390523, -0.8681194107634268, -10.305107496631964, -1.4741789352748127, -0.9123367360690767],
[-0.22735357488853483, -0.22413465060905285, 0.25079460112497864, -0.2586811303008246, 0.3890376192468775, 0.27667359302919997, 0.45073835809653906, -0.05458555460085054, -0.141080479124903, 0.019342678676835784, -0.2672043144255885, 0.0036068596108485115, -0.14725999494626377, 0.3204948127526434, -0.25373379230042703, -0.31116152624565163, -0.08777005280613039, -0.5677635034908562, -0.4255197933455279, -0.08415412116283265, -0.029849039817293886, -0.03870973173435868, -0.36931820791608055, -0.012613049509179528, -0.1856949643049279, -0.5589299871462992, -0.5495429763519318, -0.19263239745361646, -0.19347518797312765, 0.2482224757526309, 0.1954928216679405, -0.34354681602467874, -0.339824491755661, -0.6790944001973394, -0.7719718436905695, -0.21535893294957492, -0.7618359512238535, -0.5714436495895906, -1.498042276482492, -0.8771202829961433, -1.0938776921310762, -1.552968211044439],
[3.367196188130604, -3.0713192906432467, -0.6470438605466902, -1.4055389517091046, 2.0492737160925847, -2.8707302401337946, 6.39876072626098, 0.449621899867458, -1.5161933563216854, 1.9454169790456741, 1.069009744366355, 1.9345876129123192, 0.13442915904141542, 2.9531062707406925, 1.344215916178681, -0.7802539294065663, 0.8477520992308033, 0.18634987427315833, 1.5499325399101624, 1.9336485891329582, 0.5529735989751011, -0.6480151827883756, 2.2097895615569514, -0.20973989463231552, 2.532766328201467, -0.6455167187191688, 0.4967419496566559, 1.1319840287087215, 0.08276364505828977, 4.359030541494639, 0.5155081097470766, 2.5719775115441728, -0.6517282271607523, 0.5871667319218272, -0.6637802579496512, -1.4259183779069007, 0.10590159231058263, -1.7816812243950482, 4.012588154309709, -1.3588494121402783, 0.2379618055916831, -1.8023626688320498],
[0.2770571468565011, -0.3081576326326514, -0.20630343787291563, 0.3520272390685758, 0.3521022297341024, 0.3733780327161306, 0.012463944045941876, 0.4080048947066557, 0.08606541736910922, -0.4346153080433083, 0.28821411802647984, 0.14033314571650124, 0.07935251121310813, -0.06657114953696412, -0.21374453433909324, -0.19190885017726772, 0.08230696495134016, 0.348683743470388, -0.027538497149163742, 0.07519096869709208, 0.02931038686009981, 0.32555218529539875, -0.9164891457298961, 0.33743137114674565, -0.41672805203419433, -0.32205924430518973, -0.3071399895449831, -0.16608151523844406, 0.45058313291766744, 0.12261938903898834, -0.01991227889242952, -0.6157140219197387, -0.4384538786284714, -0.6302790009359418, -0.530402142240904, -0.7695723665952259, -0.9655397895450403, -0.737693372688031, -1.0300438184906981, -1.5392142922348253, -1.2344963364088792, -1.4250397957311125],
[0.3290772335651188, -0.2884330851247599, -0.573602966609693, -0.1896852988422611, -0.5339994464438558, 0.7066636174775774, 1.016391722768983, -0.007582705642716467, -0.5225169269659015, -0.7866899719928191, -0.4472780902905664, -0.31811556709003547, 0.2176602279938657, 0.9626510025194873, -0.4591822126650903, -0.47374653003537054, -0.18114354496450658, -0.20819448341446445, -0.3733872816502722, -0.43178329413960265, -0.2710006762995004, 0.3865873977026362, -0.655253699259715, -0.22408729286402634, -0.17845706940148062, -0.6775133882958874, -0.87068213781101, -0.11175020440019523, 0.07365393546569135, -0.7985083125785662, 0.21643622031027313, 0.08252064557485753, 0.029914770721445598, -1.0362965227655194, -1.2811551598751199, -2.309142883790786, -1.7081957816146816, -2.1546365981171793, -1.7194347886857857, -2.1382053767716767, -1.916683938203929, -2.5399180343593324],
[-0.19855208252542292, -0.052273963106785594, -0.0287453333203596, -0.35884081740039314, 0.006111895804143239, 0.03980885864808387, -0.08566557798620253, -0.24289486397445073, 0.18261994647395438, 0.29078706950237937, -0.24703925081516817, -0.10522690870018188, 0.12252959883971644, 0.03775714858198314, 0.15061637566017785, 0.06611051424546752, 0.21246479242600394, -0.6114175079145039, 0.005970161323227438, -0.6189767999660892, -0.06671211549269114, -0.2680904541723982, 0.23011344598900083, 0.26515016789825496, -0.24213158217975267, 0.1229676839833055, 0.5217854707078505, 0.1792398357665428, -0.3700819668689773, -0.029643108529331955, 0.7311755395923001, 0.6011061070868026, -0.18079251578581268, -0.35376121315241316, 0.015732609257304464, 0.3078179708952878, 0.16200347378711918, 1.2101822094368968, -2.017783067411629, -0.5277841602115939, -0.15694270395837168, -0.4636120348235482],
[-0.13161973338011287, 0.002437537104807592, 0.12944781452210422, -0.25307264894505993, -0.17331948459435106, -0.10914142777810429, 0.10195516316352637, -0.1377105978518035, -0.022681909004809944, -0.23798344971084548, -0.20421540663278254, -0.12802429440580554, -0.0414134217012602, 0.0868400002770213, -0.16858597448152635, -0.20878111109061728, 0.0046749031005506305, -0.3693733190165063, -0.07052987248912218, -0.35503135933156477, -0.07928129557440604, -0.050847971247892976, -0.31371808856504846, -0.4014825604581272, -0.4812222455899189, -0.21032816746468183, 0.010816376436036354, -0.23167401793610087, -0.3555171419453735, -0.6155114321524244, -0.5119046210346798, 0.014062976095573198, -0.565760924356457, -0.7633485723096899, -0.43582988900375647, -1.5726925886578136, -1.0472419610427945, -1.3072708900215775, -2.106117649449635, -1.4137261658644091, -0.8477532708673539, -1.5808670890992842],
[0.26353224117692137, 0.17036123473381115, 0.11441202302218706, -0.0275960715647341, -0.3661977843344333, -0.11928436212013158, 0.1444388912579799, 0.06684176457444711, -0.3879870223150199, 0.06994326753937852, -0.1372453377931205, -0.13567534120241795, -0.6028332484055399, 0.22211079802790615, 0.419274492884585, -0.6721675287686523, -0.2026644640214636, -0.7385251527391045, -1.3428021209995102, -0.8789790940790381, -0.4056310215719652, -0.17093112456942564, -0.5510369078819679, -0.2703561702086935, -1.4442754023747473, -0.5223038332375877, -0.9838829040547632, -0.7527544699735249, 0.014564830005318755, -1.1603550627507173, -0.6534158949160963, 0.4854067369349712, 1.2152903485662347, -0.015532419562802578, -0.8410615569844194, -1.7945399411847367, -1.9383690249376804, 0.5103196820758077, 2.3576806521252056, -0.6759739543589763, -1.4768067327643786, -1.5768556528050541],
[-0.06144227218483136, -0.09677477820484062, -0.09545795097116569, -0.4956592579021257, 0.17413340952233147, 0.13001604438670972, -0.04232601297668838, 0.07839832147155976, -0.10362658218065778, -0.2903823721686411, -0.3047694204966265, -0.012907193116491892, 0.02057958184531786, -0.21983709570424448, 0.07782970425440586, -0.15636226298981737, -0.3140630114152157, -0.3998177866553472, 0.1293660945394934, -0.45620809383914335, -0.19323483670564523, -0.11163219484157752, -0.06075537498832347, -0.1431776285237627, -0.47638251437312173, 0.1274304555213062, -0.11306243516614377, -0.28052588926968935, -0.2698108610266253, -0.4759407904673038, -0.2644253895540197, 0.35444630068624183, -0.5871520281331951, -0.6591629419593295, -0.10223820100023862, -0.2619541122257859, -0.4210000932285468, 0.07743229778447164, -2.2217035726786327, -0.44151736563065025, 0.31496813867611484, -0.6768983615096776],
[-0.06299176057830036, 0.12989291124936936, -0.1532858869174723, -0.055957453794338224, -0.18262218836317262, -0.1399765352852429, 0.26714476369150736, -0.13168986020883952, -0.017972881311898233, -0.127243095067993, -0.09029283146935299, -0.3057461487449582, -0.1885891377438936, 0.09110713480101632, -0.5593850388337638, -0.003503492176350489, -0.6507770445407737, -0.17992050963301182, -0.7507964446677361, -0.3362903747471013, -0.2035233009272232, -0.1661243737396382, -0.3475061936656215, -0.2963094381783184, -0.5871598726592524, -0.23673974640639817, -0.2759977549920943, -0.12154142773687875, 0.7745609128681324, -0.8113039269545749, -0.10600543982384718, -0.3174909131540455, -0.20492789390045793, -0.9320322002108553, -0.463403903868056, -0.3427575068609813, -0.03059433297887732, -0.3558533328955178, -1.1159822719530377, 0.21626885867504278, 0.5118073564352762, -1.4099941457433454],
[-0.41311716169752244, 0.3507503016386861, -0.28427031790042107, 0.07858309460270789, 0.6328465651254155, -0.1538856482290801, -0.04233784807814457, 0.08779715584483536, -0.024577081835422537, -0.5834148102983938, -0.33121487102337005, 0.4049592159589439, -0.24191979385410958, -0.12421122227391777, 0.07983068835027367, -0.008192937523535495, -0.3968555272104134, -0.11149867859512451, 0.1116504021686005, -0.3531604527929212, -0.15868262049418563, -0.1923607685742563, -0.11866591883691845, -0.19964970053014652, -0.515032803778626, -0.16835058416775903, -0.6479258550905609, -0.02121104475277095, -1.0014853244340964, -0.17199124474957475, -0.44259174414939645, -0.7107951889659053, -0.6381449600651796, -0.5047795570185836, -0.903447325695733, -1.118929527470029, -1.4019667466829429, -2.25088777454474, -1.813057816588649, -2.241439931463063, -2.0244013635428013, -2.529757386311887],
[-0.1129078806481279, 0.003939676418440722, -0.25602548261222985, 0.28134214615334474, 0.21391890621593995, 0.030205625656974527, -0.05705499005172317, 0.07229005722461447, -0.242693125283447, -0.3014684778945115, 0.22320180828681, 0.006225832812442639, -0.06982821565589366, -0.19142003962525897, 0.07387699774990948, -0.45689723425621703, -0.31039023703187657, 0.08459478943554871, -0.07980717907231344, -0.09461281075515468, -0.2679620316329006, -0.3544271712418909, -0.7678502781018679, -0.2974956975375529, -0.23222758672426752, -0.45110331331342707, -0.2928797062772505, -0.16720261700842612, -0.19022524126925305, -0.029326353564337822, -0.5201526601446447, -0.3328395382614643, -0.5711339852662549, -0.4573696354481645, -0.7507648617774262, -1.6189392534078626, -1.415938033052878, -1.2405173335725506, -1.638208195456437, -1.6110094631079344, -1.3136388344142282, -1.6915020418018378],
[0.13443931667543949, -0.089734687134759, -0.08607813641015553, -0.13236818058062885, 0.013772255880657832, 0.006067211951600463, 0.12351016151708305, -0.1886013482547458, -0.17561090747021482, 0.006645485532794072, -0.24494245007599857, -0.0531322338453113, 0.007703918549310478, -0.07730131587557511, -0.22282029085657717, -0.16282292355388522, -0.19717080089233174, -0.28740551364004563, -0.2981779497334166, -0.196555152212125, -0.3021935716005294, -0.12183221404932394, -0.28014563988548635, -0.4926023057048675, -0.4205242745617519, -0.3579647971414209, -0.23761977617292732, -0.1468603272840955, -0.3811723247847887, -0.6061706569233701, -0.3366993954592311, -0.16154050083367152, -0.1845963170779827, -0.565550431186094, -0.4639495172528321, -1.2207651712502705, -0.8117144383412761, -0.9297406978671817, -1.1810689308197266, -1.2716999051836422, -0.7561340327311197, -1.1870886039420214],
[-0.6054571872559275, 0.45645312808154265, -0.34947673906969845, -0.35690936467132234, 0.045814091272892134, 0.1906772838387605, -0.08589086484640378, -0.34070453804450934, 0.5104427056088019, -0.581708582394711, -0.5753881488398446, -0.16960769024369993, 0.059692285851895684, -0.21254862215413392, 0.39452693724821764, 0.6477577272411469, -1.184195581540974, -1.0126755039916462, -0.5341469526991318, 0.09875728786544417, -0.6362411556453457, -0.6747416608977699, 0.048985844865373135, 0.6588424791364238, -1.1836429463341855, -0.2012648023859995, -0.32998346996651223, -0.09650044284153969, -0.20321738075751336, 0.4245118912322746, -0.017561252240139675, -1.0116954971149028, -1.3485688866972068, -0.48500186016087954, -0.7572633836574653, 0.6310926820400284, -0.3342792829088791, 0.940270443564777, -1.9536177334019098, 0.3296699953659755, -1.391671486783211, -1.514995778719324],
[-0.18083026738701455, -0.08135721269093618, -0.19015837846560243, 0.3527125106688191, 0.1506128375637092, -0.06554042996910525, 0.2629885418189026, -0.010354485168035565, 0.020026006690279253, -0.19286906655772323, 0.1417452699500346, 0.10207190621635015, -0.3085879281780352, -0.09471669653263945, -0.17745614858886155, 0.048830799519427794, 0.030901767362110602, 0.10490269579303349, -0.028857457248559936, -0.15780949292512264, -0.0671559268131074, -0.2946856101347843, -0.40176490688913175, -0.23648675307583591, -0.26750493349612764, -0.22153727873771206, -0.4771852736998775, -0.2354598601958906, -0.44948151311463347, -0.18056398531305384, -0.11924941013749427, -0.42414517479084607, -0.4183311596225235, -0.21890065668978903, -0.5155765891182139, -0.9108020696581993, -0.9509253717876325, -1.0806462987264729, -1.1806356097402355, -1.3264427547966473, -1.2614838714847425, -1.5926920821474628],
[0.08564203235947564, 0.21741138307040528, 0.10624020208258447, -0.16314753949544603, 0.05695637968206753, -0.12013177213838962, -0.05864031794113787, -0.22543011366942078, -0.03998424073245847, -0.047403178781777036, -0.14640965982725304, -0.18244379772059838, -0.08209033396445334, -0.16156746590223417, -0.13502088877189763, -0.25740991737535174, 0.03193586082247103, -0.7056011653391616, -0.2076124427896291, -0.045894243289432354, 0.2628163630469586, -0.20548521426651048, -0.043294058626613285, -0.028184325902668127, -0.5808531469816538, -0.12290826261683983, -0.22099696238172128, -0.005351131784790151, -0.6598733703984017, -0.19377634194953733, -0.6081223029845461, -0.36852328573493076, -0.46883636979091486, -0.4892664121083906, -0.19428399206374544, -1.576767962069466, -1.0849064165729039, -1.347836111036845, -1.9347631088727097, -1.1887002856591327, -1.2826941302426655, -1.2628548760721552],
[-0.06206771992228209, -0.2630813535008987, 0.06421403849420075, -0.29535416244993945, -0.03230183232625864, -0.05857513574771914, 0.15626059961846966, 0.15249154781165472, -0.1291485996885683, -0.2791449386568846, -0.19292768654982542, -0.092484228825905, -0.3248458059646149, 0.13577142548286694, -0.19906221121887596, -0.5572090196242253, 0.01770169559157187, -0.34773870300790594, -0.17922107091034284, -0.3445622297941516, -0.13716463596983536, -0.2865744190276232, -0.6677443029081092, -0.49724966816724203, -0.24325098759387917, -0.20042392531928754, -0.5844855618882637, -0.5044646522449023, -0.598272510326008, -0.4274565396730292, -0.47415299091494767, -0.49867366775627237, -0.47093380325763173, -0.6663139977484276, -0.7655444052679594, -2.1656688183270103, -1.6737723575898182, -1.7685203467372064, -1.9988438745998838, -2.004372335911663, -1.792929492851377, -2.3305000019252233],
[-0.18323466703282895, 0.19535961008679883, 0.059631065496586484, 0.11849266438920868, -0.6537839351035093, 0.3753456351940288, -0.02965442235880575, -0.019405512374899757, 0.2308172584427219, -0.28984039963551317, -0.11468363290786229, -0.3724693784569356, 0.29627998161326774, -0.10497462752812556, -0.17663690864735712, -0.07159229752510046, -0.1582936605956896, -0.05455732458646055, -0.37971836801426173, -0.013716154662242835, 0.030866739793699208, 0.0770285160293202, -0.18215055551630727, -0.4246350335011034, -0.3437819800650221, -0.3576819994688697, -0.1328946355388014, -0.023415917719799968, -0.702854258449191, -0.36875547869641073, -0.5646334024585744, -0.2320329490708817, -0.471802100698714, -0.6782527614547224, -0.688836512034411, -2.2282281240729493, -1.9207922101595603, -2.6244564535055455, -2.3676241540174017, -2.269395634456601, -2.1691264691877774, -2.2309124267290157],
[0.13157796890472787, 0.1178513877106604, 0.0763714986751439, -0.028009012839081738, -0.05049520780378166, -0.1448693701568168, 0.09581858149166465, -0.017364714184578554, -0.2766217171039434, -0.1714728673509684, -0.13042446814619435, 0.1520858381824665, -0.0639864427990759, -0.02331745245398628, -0.01588758953566028, -0.2155087243070684, -0.22945926711482642, -0.16721004236665984, -0.0056687452021307, -0.013794589215551427, -0.2302485975472365, -0.19317996370444734, -0.6278845142781815, -0.5420332933016951, -0.5513246296721586, -0.20458733866313575, -0.5152746489425197, -0.1475437385443203, -0.5363244306698686, -0.6626994155714707, -0.11829665918931773, -0.5381682115684443, -0.17587222879253567, -0.1652790134048512, -0.6870598647717121, -2.108306078848677, -1.4155959174384276, -1.7892672404561774, -1.4169326592792886, -1.3124143708153648, -1.3340874050251104, -1.856100317707502],
[-0.093000654736835, 0.03249400061125694, 0.018271288013746324, -0.0552564943502804, -0.09441558042387287, 0.045135865340607394, -0.14109777482948094, -0.08649670694864438, -0.02315083639549692, -1.1740764033507274, -0.8404225536154682, -1.14186245022032, 0.15168552697044344, -0.02710737399602929, -0.4608159180252748, -0.8627041703198347, -0.861022436882817, -1.1823431767328085, 0.6063887372965318, -0.10368670721753637, -0.02979633195601664, -0.8339544886637691, -1.8110271484105607, -0.38582683188159866, -0.5375280608664152, 0.5947645072912263, -0.5574181532220018, -0.5533447040893636, 1.1464537944278355, -2.775665569619277, 0.4470246479654123, 2.805277881198747, -0.2864500431679287, -2.1156292200602116, -0.9699651826243003, -2.8881008524411893, 1.630620871682172, -2.0940701821909773, -1.7452438858380335, -3.210920414822731, 0.2216730260707826, -2.429582160275121],
[0.09144208580472889, 1.2879529170588566, 7.885204585312613, 2.1837175589024134, -0.5123070454834026, 3.961740989262394, -7.724206365772103, 1.538028079849119, 1.6517091302897422, 1.321890755485581, 1.4243063632859605, 1.262444392562401, 0.30586075944841906, -3.70584976416614, 0.5274386168243173, 1.160511056711804, 0.5316665653506676, -1.4628164112331883, 0.1859588108234635, 1.4687536646823127, -3.66604631503083, 0.33331829937786245, 0.13879578648993876, -0.37594765068850683, -0.3337548679312324, -0.5594882216870336, -0.7492695197980194, -1.5423348496145046, -1.8636422157549077, -0.7791477747716645, -2.7162464836348383, 0.03411088818327038, -1.123157213230195, -2.0160280581663717, 4.031057979681031, -3.082526392255288, -1.3091320369433264, -4.804667860019284, -4.5506957019719545, -2.5202170726268887, -2.5043142155489018, 0.132922662856749],
[-0.11529312874781004, -0.02106151727692645, -0.0630542485612925, -0.03548030084066301, 0.25759664195822984, -0.034252922551194685, 0.13185295289590396, 0.006940802043094008, -0.09558922301100767, -0.062427529305857696, -0.20293229382833214, 0.14165197396684312, -0.14568880562792594, -0.2372454772969089, 0.09200915162971472, -0.20753518936553333, 0.05173648476085429, -0.2945547555003103, -0.14108854201523363, -0.13565327092825682, -0.07596371839517681, -0.2010855611228051, -0.294465508157326, -0.1054986636867309, -0.5558922326347348, -0.3975471868691819, -0.28524747243431886, -0.10550528593588189, -0.8062404057019353, -0.4436195723942499, -0.36442279324187704, -0.28006347512815893, -0.6399971788848933, -0.3885304720552108, -0.33220735440425486, -0.8726410751686312, -1.0248825046399306, -0.568845817575452, -1.3858272551153958, -1.0792287903568734, -1.1101428579864332, -1.072652260311318],
[0.012486643120852898, -0.7416546932461726, -0.23377662835505364, -0.0646064754497105, 0.14816929828266298, 0.3652085836225877, 0.35545131553476006, -0.051610386558377234, -0.22648798983656887, -0.1999412426360956, -0.17597785521274584, 0.1704359463940331, 0.41071796190554855, 0.28680532236102374, 0.07896199412815201, -0.34708934239706923, -0.45287230514811855, -0.16803456766973796, -0.1958297683901672, -0.06458936343998646, -0.2856271398817414, -0.14095585443351058, -0.9443348406275781, 0.3282625394674166, -0.30058359831085163, -0.243918529219384, -0.16610576540541047, -0.5201088047180091, 0.150420895473142, -0.00325905846193446, 0.14294391412136814, -0.17978532853514742, -0.45858108251924135, -0.9572411208172327, -0.8641553425172457, -0.5951588450220172, -0.7392272532408287, 0.1869514150105011, -2.1941872828168325, -1.2235474780561129, -0.7189283200933322, -1.692580891907325],
[0.2642431464530135, 0.15715011906845774, -0.24805884010124685, 0.06707414431980208, -0.04098654553331387, -0.027863330683363968, -0.05811539695132488, 0.11125331503720855, 0.1136497637523107, -0.014076604927510917, 0.08968584365755024, -0.2300966427482443, -0.10697145424552468, -0.0416008737489976, -0.3861356675369302, 0.2308731979554261, -0.3117128922253667, 0.07112551116066067, -0.35366440313961667, 0.006025420242409216, -0.1610807553027544, 0.032196328506799356, -0.5149464837295894, -0.20395140912492665, -0.5350213977593801, -0.5341794046895123, -0.3846406016271225, -0.20747298191845673, -0.12157225666807031, -0.4815199667459894, -0.22565647456616003, -0.6538630349448998, 0.029167320690950543, -0.36290665338522793, -0.5156664387709062, -1.183320668131729, -0.7765381373466287, -0.27611852057019565, -0.7300646932692524, -0.8076579696569638, -0.4480440887959321, -0.9300920619265158],
[0.3020511529409379, -2.4522547292252153, 1.4339344607652953, 0.4978316176899049, 1.7927548833999183, 1.5546719528734467, 0.43646242673406827, -0.3077763619374723, -1.108782817669607, -0.1736751703532433, 1.755923327042088, 1.1919075487035793, 0.4824363957884464, -0.2145145562046454, 0.31904387765209785, -1.9453447507315411, -0.19372917068027226, 0.009538904797805198, 0.7240615381465322, 0.6123427012018081, 0.4290950309933437, 0.3692872777912944, -1.0864339702278398, -0.6768463210025752, 0.26143317995453125, -0.500339358061049, -0.21648136840464477, -0.30358529562399467, -0.8682822942634583, 0.7338129568078848, -1.8845399920891093, 0.7952366910720812, -0.3352911503540022, -0.7371867105537452, -0.7048351398873837, -2.8844173314078145, -2.190507881446088, -2.5130500077579208, -3.648501905509257, -2.59148475159028, -2.651734876645487, -2.4179677844708567],
[0.26876454406127775, -0.22140952932278118, 0.7638209799508398, 0.343384739192434, -0.12162630481998658, 0.3624448921289717, -0.44635220515755997, 0.7574585974642477, -0.3288257054107555, 0.5633394604086763, 0.13302365755561854, -0.12137003338621102, 0.08846773128255386, -0.08514143038898583, 0.04008449402389848, 0.19097539867571392, 0.12284649718349913, -0.2839208318291819, -0.10542116554537036, -0.045805308734836014, 0.08625161464073894, 0.05497761682832928, -0.646800729147292, -0.19394197637110644, -0.6632190718765614, -0.25987517566412954, -0.2403352993406319, -0.17153156352127458, -0.5655706656612144, -0.3234097269457375, -0.5448217735592311, -0.7218255463640983, -0.4026503185040648, -0.06015430903074007, -0.42460458711359317, -1.050497269345016, -0.9329683627455747, -0.8142262136512867, -1.4513165993676638, -1.0804034429716054, -0.757970559224337, -1.2689500994730951],
[-0.21627799109654983, 0.058305575632079484, -0.214797619647895, -0.2555359375714624, -0.07972310075964494, -0.10496121044833305, -0.0068090746126488804, -0.1818835855756797, -0.1025026374629591, -0.17787386866425692, -0.25090180753808383, -0.007633151235266683, -0.19955908046130683, 0.013333017224358297, -0.050254085202667144, -0.37476858615994246, -0.08914131380782923, -0.21061093555309188, -0.24901037177675464, -0.12331649190967359, -0.27128488303940484, -0.10751759253156916, 0.27224049730198335, -0.09526840079643807, -0.5301847527430219, 0.0402927918313123, -0.4374142857189225, 0.05385765054530984, -0.4356228593383425, -0.3830105559046782, -0.33362797163104935, -0.10617845517520186, -0.42604064220741017, -0.3641165602604034, 0.21359751624646323, -1.0773758452457083, 0.8808398963865298, -0.1258222772952857, -0.9182487270206224, 0.18969795651367843, -0.8773849030407401, -0.42212948282857804],
[0.024192223743334067, -0.16007129973209072, -0.25624686573229105, -0.24729713845742643, 0.006658596691868733, -0.1160100649942137, 0.1574208585595382, 0.06315625944252995, 0.01422322331365837, -0.33941747782094644, -0.048497979321766874, -0.12460774055614879, -0.002355754020905337, -0.1249870389009296, -0.6535422938538448, -0.2529011081563136, -0.476329984963083, -0.34657568059836563, -0.0661154864347268, -0.43895775495709005, -0.03710812207560746, 0.3564246234030685, -0.6543713046954305, 0.009418458179513922, -0.39932446923206744, -0.06669105686848123, -0.2095600381177494, -0.17688605343012215, 0.7268820120515189, -0.3032321820670354, -0.6424559943965795, -0.4024398729630117, -0.7504765281163986, -0.465420725500189, -0.5605751415336334, -0.14192665589436668, -0.7969875799352294, -1.0285835884934744, -1.4005250630257482, -0.8684968277811964, -0.04799232190477637, -1.374296312529081],
[0.06561908100746765, 0.1316311608600849, -0.4617423111972627, 0.033985903342610216, -0.016412109898656838, 0.05016601946247678, 0.06072566208234789, 0.18597977060091003, 0.06752498144911391, -0.4899974706937035, -0.2331417794703438, -0.164598390361065, -0.08817053757587691, 0.042522956778847536, -0.23051248653976697, -0.20884285825826454, -0.6982814593129977, -0.38057720230748504, -0.43303306995626056, -0.19655539825191345, -0.20094665292660208, 0.09713067727849634, -0.3234179646710646, -0.4559835960615893, -0.2571111240955699, -0.5677561573124276, -0.289084560820549, -0.19729629024507908, -0.2519333348102948, -0.8488866370304589, -0.32272702062170305, -0.49420714397718407, -0.518772990567031, -0.5530280417572206, -0.551149340111898, -1.9208387902248278, -1.4872567388323212, -1.5250735871252175, -1.5045092028737765, -1.7156997445951088, -1.5754245211447275, -1.6716708711313126],
[-0.06602327337711698, -0.2534806273531349, 0.21192984090541833, 0.4662772561993441, -0.06530454405084643, 0.24129005730041514, -0.04207112036876581, -0.05837368596324695, -0.21193122082897467, -0.0570590359770316, 0.07250594981058756, -0.2411657726912715, 0.3171822931468101, -0.1758855685534693, -0.03825053812010398, -0.21191891405017313, -0.17853100791314147, 0.1326943448398987, -0.3099312079873339, -0.16268267005293302, -0.1163793398578545, -0.17941897078883193, -0.4029384917479725, -0.3491903522705163, -0.059135875049811944, -0.44654221624177964, -0.3017839063886506, -0.40949272689770255, -0.691513985388484, -0.25979223014118874, -0.5262695809854422, -0.37173663098882137, -0.3987139731172355, -0.4902951590163817, -0.6594036229166189, -2.777933977316168, -2.4462640597302197, -2.8284088414796162, -2.7279698069218186, -2.873521901617778, -2.774159079436968, -2.8683967155008303],
[0.1129207123650427, -0.10259971919130267, 0.17914646256780956, -0.08915551458999534, -0.04559765466610918, 0.0544687302918998, 0.28548277064952404, 0.22430891043782777, -0.11940678746740752, -0.08057043701909722, -0.3124482249767151, -0.015579385910517987, -0.32093996944546166, 0.15827262837701117, 0.054134396900793184, -0.11545948728341471, -0.23535129369304605, -0.44246246006038914, -0.053044373708314774, -0.2847088190658418, -0.16322393499165525, -0.5477398634277658, -0.4643931246334993, -0.18684521416484834, -0.6257552741674336, -0.3181362977542995, -0.7079397768423359, -0.37667444123352356, -0.8863576722339014, -0.42743784892303655, -0.6651662075031323, -0.5116863830285299, -0.6071147090419905, -0.32071238671543667, -0.5617240199113579, -2.787654313434452, -2.1856462053829837, -2.7439099403901497, -2.9013034630585093, -2.716435518649575, -2.509702607178054, -2.8802270233667913],
[0.013315352064383886, 0.07543210669046552, -0.19098767993652407, -0.04324356811463036, -0.024922291263937177, -0.11751502243106557, -0.10437829755622723, 0.15668902954373867, 0.07551656742086756, 0.0027313232306407412, -0.344421007179513, 0.17244942432680974, -0.2071609284643225, -0.0808838320831152, -0.2752102267683253, -0.27022797142981503, -0.3049267359094769, -0.4598306422216979, -0.11027710141301857, -0.43896436856128096, -0.24112478456160108, 0.04734507234847006, -0.5033818746000257, -0.4736115994244247, -0.4186769967608233, -0.17311640060571276, -0.32286331513527067, -0.15756071069941047, -0.23437739687736947, -0.7337472937097599, -0.38261758147007896, -0.1573843197467137, -0.27714691716364515, -0.9048983146875447, -0.5451530667319398, -1.8773063574934628, -1.3334702191846735, -1.6649979916756454, -1.8319924006384198, -1.612327483141501, -1.395232978145396, -1.7061134888106761],
[0.033553379950039486, -0.0157432806386093, 0.24433150345296475, 0.24330237015585365, -0.2568011755351491, 0.5489717033740058, -0.2769964509068257, 0.2701294109805769, -0.08655099153604819, 0.11687313159810064, -0.1719515367020153, -0.019485104228556573, 0.5293910545783603, -0.2286079053847341, -0.049649740447610155, -0.09329964561073087, -0.14067660527829853, -0.28822196125872923, -0.5000317594312953, 0.09573626584557514, 0.05332936993036564, -0.018934101266473363, -0.31917125838047467, -0.17307506842872447, 0.029098106661768965, -0.7721133369342261, 0.06993434881507252, -0.17966830770299916, -0.8038633818815067, -0.6274129321465566, -0.6901156937649122, -0.4590973358702765, -0.6409326548454203, -0.5222489089047876, -0.7045133006230323, -2.1971811515976283, -2.0211772958027767, -2.142710894269146, -2.494872579002953, -2.5148155278756126, -2.154768144400963, -1.9683584693305216],
[-0.05462417187667894, -0.19170004951632008, 0.26106892104857776, -0.14147390674755933, 0.13838735794490914, -0.2946194622242232, 0.04417902197312572, -0.07101508316015086, 0.07519597541906328, 0.19126243242071797, -0.28952264989754783, -0.15480846121278471, -0.22493611377601389, -0.04646244364190904, -0.20365110981858955, -0.11904677099391406, 0.07221064306513379, -0.2073886592003876, -0.22096630670577386, -0.031240348600645593, -0.16065195380592479, -0.26626434985097114, -0.3574794538253473, -0.174327878508239, -0.45378356000813297, -0.08344147274668105, -0.2566711124828069, -0.3628273687421782, -0.7997224841268992, -0.20619415078569686, -0.7253388936805494, -0.45189867107766524, -0.40570391437197184, -0.0940091638210684, -0.5866644625648861, -1.8337726474924356, -1.454232721130518, -1.5041868711501838, -2.231207271754233, -1.6208673407123833, -1.4324393994726143, -1.8596988282433267],
[0.27119784166887084, 0.31454115219595974, -0.04435771002064374, 0.16142884336245358, -0.15344727981125592, 0.431762042248073, -0.12253261707881423, 0.1639362174981054, -0.31967292177548584, -0.48625107372399545, -0.12868531815647516, -0.06929331527420854, 0.010143996452053551, -0.11809189507227014, -0.14237307931108276, 0.03422158382563074, -0.32013832867968867, 0.26079162417658724, -0.5500861084310376, -0.5179244552619353, 0.0255174066243784, -0.3665270637295454, -0.3410779377351099, -0.07574647824563674, 0.007189055506228017, -0.4858584571570792, -0.3169392995153039, -0.4905422096832255, -0.6657528698376624, -0.5177800700186003, -0.43908984969552595, -0.23678036512280942, -0.8519907661277456, -0.4971153713184848, -0.8184856210434261, -1.9631396565211072, -1.9615495939336476, -2.0492100301349536, -2.3460811729256, -2.0948615311731063, -1.8005850926059639, -2.1017319318431413],
[5.007424546297214, 1.1378002793861763, 0.9483207589295194, 2.2294164405978374, -2.454505267191864, -1.3525943623702836, 3.2111067622242166, 2.203574544325705, 1.5771970635650485, -0.8087640990140527, -0.21216249971688703, -1.462743760172417, -1.1508076236734497, 2.0948495291321203, -0.942967149374593, 0.3108068962853128, 3.001037561500823, 1.4284752317476574, -3.930996321508056, -1.5735836850319977, 0.3470543506524054, -0.34715883630466254, -1.7108630052685487, 2.137970840951424, 0.6186289665954944, -2.9269059847538594, -0.8214137424171388, 1.2389248459257811, -2.200595311825784, -2.033597299195871, 0.9706761529923807, 3.851429642156681, 0.8319245428696098, 1.7848941708396253, -1.2059712582948685, -4.407948463469599, -1.2824102805777615, -2.6477974596864087, -1.7891972299605934, 2.1903623513945516, -0.20498530645400784, -2.7582676171786753],
[-0.03900597276791964, 0.3934460388889435, -0.22457849555495868, 0.3475765405677595, 0.3629004012942838, 0.10707480009889739, -0.0150714964082761, -0.24345950792459597, -0.14110798556715023, 0.02973832244152749, 0.15830807985872397, -0.09372136709503992, -0.19342475187771238, -0.010850242029670092, -0.12154827666375403, 0.2224476581709509, -0.3777071131988879, 0.3444490547431898, -0.30183368683550876, -0.1669565668578093, 0.0046898787216978636, -0.43968467272610884, -0.05346043640899888, -0.6131968656561089, -0.10208149309893454, -0.7753769406496043, -0.5939989218475091, -0.01300044652495138, -0.8008443050139912, -0.22511258693942462, -0.5282995882134467, -0.7852373700410618, -0.17160262111028826, -0.16778374080497624, -0.5627922703308298, -1.2477143726264999, -1.7785405356515858, -2.3439972597315064, -1.663207604356712, -2.3906295692977784, -1.9155936597235526, -2.294548046743115],
[0.2012421801713787, 0.3793321639748062, 0.44317937856389134, -0.5079901378442467, 0.1041453429599128, -0.22304386210467755, -0.20718440172939892, 0.30405006263470025, 0.07294232353425782, 0.29779533329472113, -0.0810093927068422, -0.20826701745234877, -0.6226295020354135, -0.28485081439420007, 0.3101033796013728, -0.3190914079490786, 0.04987993997831377, -0.08169839473179304, -0.33240573321079064, -0.47953171823921437, -0.1561066573071576, -0.3993358727377927, -0.6482820542673127, -0.2565750650711008, -0.7157803821065915, -0.16652104957944486, -0.50303418607936, -0.17297416139422916, -0.9358163971330045, -1.0105588383511874, -0.5410859691954321, 0.0027254461840726077, -0.0843012980974312, -0.8255567774298184, 0.4354835821703398, -1.5459760826750713, -1.2639139538190298, -1.3571115027963714, -2.5853682065709958, -0.4451996455473258, -1.2565699706897058, -0.7057532109849916],
[0.8628022531360882, -0.797166856596298, 1.285457485536845, 0.27611659962785534, 0.21036261259097314, -0.11317770567679694, 0.41371512759157897, -0.31509090306994775, -1.139920463244001, -0.21593703828249025, -0.7537248015721851, -0.3404843561113741, 0.07077940494393761, 0.9209294748889465, -0.08048162757580471, -0.631651758806995, -0.4136959695743894, -0.9532933459955671, -0.42428536084141105, -0.2265639086742915, -0.03959637043995115, -0.5447719930622971, -0.4264483736490988, -0.8373290746745137, -0.6910945545270987, -0.7939980930144176, -0.3267270912043655, -0.09566456865650556, -0.9509901107825144, -0.2980986916473858, -0.2632159190820035, -0.6480397495971632, -0.49068982147893053, -0.5252215889089024, -1.1223162390665358, -4.618720490817906, -4.17432789047996, -4.591781418862662, -4.436214146585433, -4.576945091985951, -4.354413443261565, -4.614515236823108],
[-0.06648074698622718, 0.38674965781731263, -0.17857522009573745, 0.8912997173413937, 0.3659606660867342, -0.032651412130459787, -0.24529449140866186, -0.8151008702991599, -0.3405261954926507, -0.14950795995210742, 0.46317895372396006, -0.10045718790590678, 0.007400258254301878, -1.0836907039080275, -0.21559593071373542, 0.4074994191503367, -1.1717277367096357, 2.760515285049101, -0.631043277923003, -0.30766710646299006, -0.11851897484563584, -0.35832488517762595, 0.03907740212233239, -0.6383479445972711, 1.115275339560608, -1.4075191957878628, 0.23404432320798446, 0.06226422423263816, -1.1854865264198853, -0.35972959842016894, -1.7562410964480617, -1.2342818232898067, -1.5496599664555675, 0.4579042719480672, -0.6320404621700106, 0.15559278163941692, -1.6067742572753039, -2.4140593918277387, -0.08851335369253166, -3.105693526394101, -1.8910002825955647, -0.9844791362217188],
[-0.24573464936974496, -0.06929168678862208, 0.13321987920656728, 0.251094843606843, -0.3530437100029342, 0.012344865709002254, -0.015580318014138905, -0.017349438313696355, -0.15276744028094802, -0.010532363385171396, 0.2183648888034152, -0.3370063871249864, -0.11849669563721699, 0.02399320789276655, -0.25434944919856045, -0.016593460331293462, -0.10814719856949925, -0.18124036151436368, -0.3838002224772864, -0.14570538507340391, -0.3079849133718712, -0.18289867702739415, -0.26026788288823455, -0.07861844330159468, 0.038740032836670026, -0.8805571301171052, -0.3808159214177543, 0.20347876021961778, -0.71959300246415, -0.537013126222612, -0.5511913921929606, -0.286820293772603, -0.2035945332591219, -0.7338992915813713, -0.6050920906371757, -1.8965782175624268, -1.620032599929442, -1.8972278929840836, -2.0306069249749807, -1.6315384888166897, -1.657767954452094, -1.0934164407393303],
[-0.22944349259501373, 0.010749066345966889, -0.2505041826455357, 0.3276536978846833, -0.07255626515961099, -0.13961346643369923, 0.30431529182115713, -0.05545425288307845, 0.005040390379735332, -0.17020625312677481, -0.04737777957720759, -0.25370527254981484, -0.24421853468621335, 0.026659554913590164, -0.18734523130639325, -0.018422773094307357, -0.216674915031965, 0.012611354513526511, -0.08103627010003922, -0.20972497490773773, -0.0009826705161150345, -0.13221619426454956, -0.31001340860386917, -0.36495716897607267, -0.08540142099189038, -0.4672133796328, -0.5234704776415167, -0.14683624649855598, -0.45402218444979936, -0.44485949187589924, -0.2062870319333241, -0.6686199637007256, -0.24259101591155, -0.3418303562199564, -0.7370116757618712, -1.173675867242761, -1.1661564944795635, -1.6820796406984755, -0.8945292957084473, -1.6314517863475688, -1.5827709120067202, -1.5041689733592396],
[-0.19502965365077887, -0.19607639397839316, 0.07575174992062059, 0.11520261343705963, 0.41157535759708147, 0.05869875180959613, -0.22101045519109816, -0.12513530136671108, -0.01766783335216836, -0.08376594283219363, 0.0011267131119123506, 0.15077632425058116, 0.1047040601557241, -0.025514939567324188, -0.11602807696800721, -0.2127290452205815, -0.04752618203842148, -0.14083299690270265, 0.18867052007160365, -0.14399134378920175, -0.12228791343797825, -0.13234656681436816, -0.2727846791191608, -0.18932175270059554, -0.1931647818066485, -0.20767448721849086, -0.24430120964386515, -0.2267091913535759, -0.7206719912143257, -0.33325550684797645, -0.4334751152299935, -0.35599113865774307, -0.5651194273527737, -0.5992128667609978, -0.7580570016450076, -2.326555633399306, -2.074900061582975, -2.2683363175312956, -2.5472714775121923, -2.624469215777978, -2.26181218275211, -2.559614590578928],
[-0.1514337815048371, -0.15385471295140324, 0.1353648907303359, -0.01963619325533793, -0.12202915858788517, 0.14010187079470823, -0.1299525817661324, -0.14603548177982972, 0.02115890148704811, 0.07661072495822831, -0.1359120840142728, 0.04777489630232258, 0.021973437341516924, 0.019259185753855785, -0.12101513579278382, -0.16455301481346424, -0.010760383112406147, -0.17187769237038322, 0.1356742654905689, -0.03035367168260469, 0.06646647571014484, -0.02193409527137667, 0.10510359451785413, -0.07575942934235542, -0.29001046991817053, 0.159231612358982, -0.1926836260645701, -0.03449116614330932, -0.3442740264832085, 0.10213904831520366, -0.04870625278602184, -0.14218327806684297, -0.1312485293765658, -0.19482816299681396, -0.22622264301729209, -0.04371795139155349, 0.3208882457067953, -0.26862222636448707, -0.8389361469986906, -0.05274278223912705, -0.419100802309932, -0.6785811261862121],
[0.04731596742347376, -0.07177239916567839, -0.28471879443701, -0.29039943786960704, 0.19645683971352054, -0.04952220947291128, 0.07363969579907588, 0.012152887812459595, 0.02771011101077621, -0.4700861126156124, -0.11208474398486329, 0.012574015133638674, 0.13755161077658176, 0.016377864752158234, -0.27900935637399765, -0.476706523486407, -0.25993092822301356, -0.3714894564330011, 0.30894261576275034, -0.545033814569737, -0.22356401055701153, 0.02120648517341106, 0.028535455440230742, -0.6115649252897337, -0.4854209818637124, 0.2961878001134954, -0.14369802892895972, -0.25694828803733105, 0.5873399656027544, -0.7803517678002199, -0.08462199626146785, 0.15316883622425698, -0.23159103531351374, -0.7031957607703196, -0.6386864211708632, -0.2878000666491339, -0.2567125435735516, -0.7584638545081096, -0.706981448508539, -0.9371017130212909, 0.19423210869459298, -1.3806806235410565],
[-0.0763199869714117, -0.1531836133895463, -0.13717785022827705, -0.25385253097247457, -0.15395630139518035, 0.09243008058476658, -0.04314449611069035, 0.015828694440377034, -0.5053084196037155, -0.2266532815069923, -0.2622370574071765, -0.13555374979463863, 0.5868777385663126, -0.15537043022429856, -0.08594144427320037, -0.34726104550850573, -0.16537717728724746, -0.618853061100052, -0.8657103085243342, -0.8039896859317002, 0.29190068737274705, 0.060309877835466444, -0.5101281827044168, 0.39674462066530486, -0.7643944114384317, -0.5145828319412448, 1.207376382836901, 0.3573988503378626, -0.46101317703511496, -1.0839301208384726, -0.5180020704189956, -1.5407489643124772, -0.420369131768825, 1.532898517721333, 0.020514493111189636, -0.6928912849133505, -0.6172320074588162, -0.8546161303949267, -1.5267130967461349, -1.0937113666190932, 0.8914183801054101, -0.6754244619660552],
[0.39588002869450695, 0.12761290307626194, 0.39498845010822914, 0.2120132791637351, -0.2570162625653228, -0.10686153627564887, 0.12904049128550663, 0.022783215484538405, -0.32874649826316044, 0.045470241751498774, -0.1257022568242184, -0.1549763623848011, -0.24698643674031512, 0.06307380393615325, -0.4584798337635877, -0.0759014850778124, -0.31431073336033144, -0.1309150465548653, -0.6164508578912941, -0.07336933274641683, -0.150980464579105, -0.24488336393341775, -0.37255176871336826, -0.3079805070166726, -0.5691072453453562, -0.5464454610751694, -0.48440484984927684, -0.2100173417567483, -0.7281517791305381, -0.6238597594642089, -0.6882535037650356, -0.29887997987807613, -0.1358318972252168, -0.3547215239129954, -0.6233778248776269, -1.918259770317803, -1.226472576060433, -1.3075344513084255, -1.653946818055275, -1.0056576854685801, -1.1779947957121382, -1.0815347461758622],
[-0.2785213270106117, -0.2465310557007645, 0.16280777289195086, -0.49377965048641126, -0.32175936205036026, -0.29044832172661694, 0.05157422686123759, -0.20211461126824817, -0.2178149181809636, -0.05792523832752425, -0.4938973513241083, -0.12746961459476433, -0.29572998663215666, -0.36048475479137304, -0.4108869441601125, -0.3361687632620913, -0.7735960020492683, -0.5464662824257196, -0.873369163000062, -0.4752456452005285, -0.8377691202617433, -0.5334391850095583, -1.226831963835132, -0.8015912938902948, -0.6947970164923467, -0.8884684919322454, -0.8970942573971763, -1.012736442616473, -0.10323275925512235, -1.316526431810295, -0.3382130207654297, -0.24488122939392268, -0.2727288864239923, -0.27072183160386, 0.39636260201320606, 0.00829650518874637, -0.23322791170858667, 0.43318806871577914, -1.945474558221638, 1.6647412789449254, -0.1682271266217433, 2.0164763498527996],
[0.03726362581793572, -0.012173694222269441, 0.1411202028209187, -0.12078501161359334, 0.1497205641545915, -0.1495718130019357, 0.05297943593937543, -0.10460726917288167, 0.04980008993335996, 0.3143498896985015, 0.02888939323787707, 0.2728436489372209, 0.2486584798164413, 0.15735493899276656, 0.22332597553085187, -0.041139613606278415, 0.2974987130362116, 0.18332615564551727, 0.4017875639089339, 0.2873536536291762, 0.15038111421026232, 0.21594282328379374, 0.046135944841964184, 0.5317588602602658, 0.028106912625025633, 0.6169053164362812, 0.14345998147166605, 0.08491371959352248, -0.17029570518116308, 0.03656326219112273, 0.34381138974406394, 0.3560458639889768, 0.36574602250831645, 0.2779392495257934, 0.41819041255023137, -0.3903970917633384, 0.14200507799352854, 0.7526502400414581, -0.6673203264904958, 0.5734290414498766, 0.28394240833069845, 0.04929545485636467],
[-0.1252825378114915, 0.18180755100767743, 0.15501069671276588, -0.19112598382292156, -0.15405405421681284, 0.05928151992510914, -0.11748317531556582, 0.04954026141645146, -0.01773614807972782, -0.04955515315556205, -0.30692409808193283, -0.30621735313113685, -0.08994378017863232, 0.005836287575790963, -0.05709423945408989, -0.15946104701434713, 0.16828650855305172, -0.2648331095163811, -0.1683428388115476, -0.1760651069203801, 0.13286632647464156, -0.18749384261056054, -0.15282293266827243, -0.16193556415983984, -0.09589431768395805, -0.5100941464622536, 0.09436557589400119, -0.1778063358769029, -0.7767884580054886, -0.47485943208993814, -0.4966752008023782, 0.03635774847288228, -0.6835652680154584, -0.27715322096775274, -0.577846429709181, -1.118111567413638, -1.1297848016706271, -1.1596898354501555, -1.9015517507211985, -1.2609907934784146, -1.0874421288818334, -1.2210203116797906],
[0.04457484753064498, -0.08552150870759012, -0.013165453080043623, -0.08510973546801252, -0.3955658720413644, 0.19475293030300925, 0.09098066810591642, -0.17605117625508845, 0.1127747245476741, -0.01288623380662333, -0.2740850416527734, -0.0699291657227121, -0.023791101732094522, 0.07903170249954461, -0.07047042012858666, -0.022208887616059568, -0.31110040017315455, -0.3828250662838431, -0.29716921435082394, -0.04147827004473954, -0.1410955072971428, -0.10339575291653443, -0.19308770292265806, -0.28702978832182213, -0.5232236481246803, -0.24229606174360196, -0.3062356845782837, -0.29120455881231944, -0.1940498922667173, -0.4920042330403995, -0.2784787495519133, -0.17184861592496206, -0.04312192139036559, -0.7030558012095945, -0.07987873915644221, -1.5571413536389005, -0.5008865999616269, -1.3593360513299446, -1.6805172084950815, -0.8906013340010198, -0.8486336192358712, -0.3980968922623626],
[0.12594780260697933, -0.6691225491559548, 0.2480467547659914, 0.3115811727032356, -0.39900151040327136, -0.006585013420969014, 0.5172382221360059, -0.07650109369026022, -0.46246609259342397, 0.13818959370622688, 0.002055161133631324, -0.3318732525238558, -0.1379581997758873, 0.005033776036284902, -0.056031466772978634, -0.20951473523664396, 0.23616929077587837, -0.12654469114908515, -0.20364146912369904, -0.15471730165243233, -0.002490065778774257, -0.21696500874666586, -0.6114585252118243, -0.07823565042398238, -0.2464523629633121, -0.21554377138223355, -0.32684423807067275, -0.38397788436787456, -0.7065141476147664, -0.07128403455628635, -0.14626707324053156, -0.38459845875165954, -0.3980425612393984, -0.5724385383316726, -0.7099503920621659, -1.9913894202952476, -1.7701542657525717, -1.0248755292290639, -2.371024716494502, -2.0338278702390986, -1.834165675088437, -2.1598543864281816],
[-0.08854320552344346, -0.29213352298900397, -0.273319467784778, -0.491751782643087, 0.24388983140543002, 0.31880881926863597, 0.3266026882115597, -0.2168936985404135, -0.2513295306738096, -0.4166803388139038, -0.471759081810698, 0.06234444445939831, 0.2048143684615497, 0.27289125459589175, -0.10642593805477144, -0.5485365960873202, -0.18200514107675514, -0.3833587018528916, 0.5043440381623592, -0.3933454770762224, -0.19384432089704742, -0.26584876849724925, -0.5011372346901288, 0.12755856211971436, -0.3720404767887388, -0.025091765443681992, -0.19821750320139295, -0.08113878845179341, 0.032418612833415364, -0.6552614167675268, -0.5092735760900841, 0.40958159640851943, -0.7063003279226173, -1.0890385760173045, -0.293711904656741, 0.05424746929781395, -0.35549081785170855, -0.8264281838125499, -2.5164773851957896, -0.8807919261011636, -0.046364669566779544, -1.1365787003949963],
[0.16518790495583868, -0.3298847049160798, -0.2674278993314386, 0.17722508067456239, 0.15458237206963657, -0.16132670714952704, -0.30469929246517774, 0.2231858642542957, -0.31511015030465095, -0.5044904614144345, -0.1369247698953707, 0.046302611540003105, -0.521015401750923, -0.5454510774642615, 0.0339924928516999, -0.44002829030570506, -0.08204105387206309, -0.012707488124838805, 0.6222791191454633, -0.352056460108305, -0.5017382021692373, -0.43024138157352726, -1.007424454847146, 0.05910326956626628, -0.2013485322215839, 0.08917067745572202, -0.7205899793640858, -0.2723008765420336, -0.8741204257004942, -0.0925739340450377, 0.03690354168635003, -0.7020737238855411, 0.13118638867422838, -0.19265834076267366, -0.13458235063353408, -2.1277388895412184, -1.360066937767801, -0.8874074618432699, -1.7589304268333474, -0.8130408847660286, -1.089260285126086, -1.7918459379978393],
[0.5222055142528096, -0.07409713922970913, -0.29570632620824694, 1.5575242001695717, -0.8314104154127704, 0.35472003417494863, 0.7219925677719055, 0.9609361162822441, -0.5685523849397304, -0.798097165883804, 1.3962811433357976, -0.3783673700511255, 0.11205305688491758, 0.2756344198388654, 0.5878136862692497, 1.1664373396660914, -0.12445939910347696, 2.5581628541512975, -0.7546874537332635, 0.6306355524305169, 0.1504925359328186, -0.1284673565482903, 0.02410718608188925, -0.5436617101629985, 0.5745829925958004, -0.8921702589155714, -0.0501425292571939, -0.0886788167081173, -0.22150206021484273, 0.9529621686213956, 0.7529284335076988, 2.641721798233871, 0.46511079355976076, 0.0029668295307824157, 1.0920063972743148, -1.5798561573981342, 0.20441170473195083, -1.3879494480173968, 2.7388711950131914, -2.2862177293887136, -2.345344458544346, 0.6544664842534614],
[-0.12398019197889562, -0.09010266528267981, 0.21577433373669913, -0.13166247786877167, 0.12258945641541605, -0.16177298804075815, 0.010991209324544756, -0.3551247278822376, -0.21216534490091613, -0.014407160364292635, -0.5105508397516881, -0.1434720766647738, -0.23935797566839503, 0.10116027417899699, -0.308096964987025, 0.28458104174836035, 0.04226409296291696, -0.2645448951376646, -0.021060174995125233, -0.2587811287885342, -0.13478956390174712, -0.2570543961761899, -0.18817403885200312, -0.09588655367994724, -0.6217036227072605, -0.4782688081420672, -0.44687597900796694, -0.12286387668006395, -0.7374006558641775, -0.036074469892375824, -0.5791510588697496, -0.27463637803648566, -0.7019371648313147, -0.4967002838629952, -0.504713955440933, -1.3751106827242516, -0.9126773735230043, -1.5128938336058118, -1.907667274055976, -1.7176102699982052, -1.5419227288434285, -2.0072504194408585],
[-0.008565608699373884, -0.027199270395736393, 0.04857615633631249, -0.20292682978293872, 0.0010887232305551353, -0.042583221777513953, -0.3240188036832793, -0.17443277590120526, -0.28382115200082064, 0.1687528240188321, -0.2910983130907482, 0.024666012493927667, 0.023642387952904183, -0.32994359933522194, -0.04748715551266159, 0.20432181775463887, 0.006449336752487829, -0.1441685102737614, -0.18889546819474826, 0.2986092310953302, -0.11637175234072947, -0.6944827159328905, -0.5158804461528044, -0.0481416613005907, -0.5139756749541712, -0.5205817695574781, -0.1426804377959055, -0.4049841528006131, -0.6819746128367562, -0.0016379008862742628, -0.7603723758699049, -0.7095689120493666, -0.5430756428297544, -0.595408185274582, -0.5435634731283883, -1.3503929518995477, -0.971678386977947, -0.8586272248977298, -1.640114242483915, -1.5299186291308904, -0.9902288135176818, -1.3290151661492278],
[0.2084392351658238, -0.030754084931140624, 0.05415623361501372, 0.08120314382119782, 0.10776897525141547, 0.08681014957657078, 0.011952741847022334, -0.012984027364103415, -0.020637539766603683, 0.04099069362036618, 0.05415782986900477, 0.009840007436426811, -0.10467237309779379, -0.24099486205147122, -0.08089690093806758, -0.24684379867255823, -0.0845352325167264, -0.12903827804595758, -0.08373334499252105, -0.27975954929041974, 0.010301275839574978, -0.19743289373947884, -0.23052482964566884, -0.1540602056124641, -0.33727401164281157, -0.1352076447257404, -0.291012979103978, -0.30284701087197113, -0.6194802573657415, -0.37261686570477964, -0.4014777016622567, -0.47417352723363637, -0.5762099799173251, -0.28044966938481247, -0.7124356407675391, -2.060723181901347, -1.756154100165279, -1.963219079535383, -2.321595564194841, -2.190510979038401, -1.8109266180613346, -1.9705322778068053],
[-0.1237880281865999, -0.16185867439571594, -0.02001764653486768, -0.4616565689875395, 0.31603800859984954, -0.016517083243101714, -0.1894913914499905, -0.33217453817519693, 0.08615517836389909, -0.03480328797386694, -0.43351537441164273, -0.017598520907347502, -0.09731071956364515, -0.29537889631884356, 0.37328415354407407, -0.008979227870195006, 0.2307961152023435, -0.46378333306734304, 0.1622198973766661, -0.3312824504331648, -0.09635402108007156, 0.024950709319927832, -0.3983715510760006, -0.35719217134411224, -0.41725693080913684, -0.02811142866659872, -0.2434267442097537, -0.08995999738855943, -0.968272359430031, -0.45189271961794375, -0.1937502944965643, -0.1640932014410504, -0.6389288781088146, -0.8748231892131001, -0.5029122743990238, -0.06743662868235036, -0.8371353731501588, -0.2221395248251992, -1.7303682262034288, -1.198844483609648, -0.9853915965690018, -1.3013639094909981],
[0.03421121205764331, -0.002577992622062039, -0.11210860656111103, 0.05249830078493101, 0.10630179322238659, -0.15622777317963843, -0.027997883713174302, 0.041841131517556716, -0.1866539517588567, 0.004351558112092149, -0.0954649945662686, -0.17051899295788203, 0.07516160102330999, -0.10380159323701295, 0.010186254183506358, -0.21240886681722265, -0.21288359989315664, -0.25386715541280025, 0.0012609016369738264, -0.17476358696227673, -0.26125980964609696, -0.27649704388975044, -0.20767300804587852, -0.08685902840869936, -0.6073364818802782, -0.38914389784118625, -0.34289954851764787, -0.11490818599280922, -0.5619095900911932, -0.49641913862245035, -0.28543821813575254, -0.21114086097752494, -0.3572258746357113, -0.4625053943798244, -0.6756287337652709, -1.2702102507138227, -1.1368460808514869, -1.2264602624990806, -1.762085877027948, -1.1971506592165273, -1.049276375352573, -1.3063940740111817],
[0.20175132234187068, -1.0364636653080674, -0.011418049734434116, 0.15582118534681205, 0.8427822800311493, 0.06892739271696452, 0.8439836362254641, 0.1771757077548632, -0.5880654595347896, -0.4306713808011169, 0.15819741259580025, 0.859083124997719, 0.35564749600557927, 0.19381222638476595, -0.014425708173289323, -0.8304335737752045, 0.20810701141852073, 0.110745423946459, 0.7184178057896192, -0.2579376645336904, -0.11461856181519117, -0.12300871353489004, -0.7450808653470331, -0.09412577473204768, -0.2836342445873948, 0.21751602282694682, -0.45280920670415203, -0.47950756986698223, -0.2927244679590569, 0.0649330114543691, -0.16341644554768742, -0.163988804053327, -0.6459574700335996, -0.8495346623510021, -0.7750287647765369, -1.1060600313426494, -0.8260584270108048, -1.097487590592521, -1.5756147051902452, -1.4241422769845264, -1.5106271373748157, -2.5681740092350775],
[-0.11473683091017396, 0.02392806553533296, -0.03333046333661096, -0.25665773699481703, -0.1130083063287562, 0.3780706154383519, -0.3750948364313666, -0.16386666116406998, -0.06968801032697977, -0.03312396788962511, -0.14146832129472056, -0.0719224779942314, 0.21978826884821778, -0.1887877461176846, -0.18335623323853614, -0.08186485239911379, 0.011912061388661785, 0.020424362522815966, -0.08316577728382227, -0.1282719636754868, -0.09778007442035375, -0.01516872756647784, 0.12904646678341034, -0.05868336923040997, -0.29222575782640436, -0.11558267741392984, -0.1699711641870726, -0.02280825421777045, -0.6238315328175844, -0.2553338303120689, -0.5611082502186268, -0.08658758333248628, -0.6065054224658489, -0.5529284731392919, -0.6710225700243422, -0.8935270971621142, -1.3045386276990654, -1.5361862866188047, -2.377097703396441, -1.8526874783570444, -1.2256121680807968, -1.5366708860999645],
[0.18384370813806725, 0.015501375042588805, 0.19252238156131377, -0.5761472948397651, 0.4284657781026742, -0.15654607166351767, -0.1962773599030538, 0.04206088713386398, -0.011556219790625611, 0.24747683667206868, -0.3491877644501501, 0.12819853555577884, -0.32930046547629843, -0.3074874489456061, -0.017884534407935863, -0.29417139996782027, 0.22263833450153173, -0.48101300319633766, 0.13590982404746332, -0.35927352856652506, -0.16454568661337315, 0.036531996281064814, -0.08174963995178164, -0.5944144809766428, -0.7396390759238901, 0.2986425762613241, -0.422445320881783, -0.44102133531923926, -0.7968276678566897, -0.575753192004638, -0.5260958326780071, -0.19985742215552646, -0.057101958284334835, -0.46392828619641924, 0.03415114759625143, -1.511309540161485, -0.5625263882074887, -1.5773796762492165, -2.060593741289547, 0.1509146089513126, -0.8198711567019854, -0.7393891872990402],
[0.1528544111037656, -0.04279069342365847, -0.11844627631427848, -0.41268503891516195, -0.16690976902780807, 0.011231721649135528, 0.020022887503391495, -0.06043166391723888, -0.12521624141405974, -0.10608111360545273, -0.20524225163651733, -0.20196354042411455, 0.09806211660079542, -0.12261745511294607, -0.042520472820289125, -0.4800834858256395, 0.18156452886671895, -0.5318754370652954, -0.20782317554800728, -0.2801460436421006, -0.2167658827067864, -0.3262892940988947, -0.3928120261041387, -0.19260908917806502, -0.6651298009314736, 0.020521654581210816, -0.2615057543626333, -0.2552510253907828, -0.7778587075923976, -0.560313498911112, -0.5924428596832088, -0.25103336952666466, -0.4831861800385417, -0.6603119389856991, -0.768248985041996, -2.3986188185707027, -2.116930743875921, -1.6598401588806113, -2.894079353761489, -2.0133666173368665, -2.006936259750524, -2.3464820877231145],
[0.17009650450859842, -0.10422292190125372, 0.08284734458583431, -0.21371551902623434, -0.1132482636282335, -0.20641581864697503, 0.30367538999951155, -0.0007504513638220067, -0.32267325818737175, -0.017044970741280044, -0.5295539893420738, 0.31294630905871157, -0.13394116711178625, -0.19791526486970898, 0.12215073464678522, -0.32832295816966367, 0.1533953416594709, -0.5447633453703692, 0.19257052881700062, -0.5390225696244046, 0.09630111434045613, -0.001611216669039855, -0.4196163842013964, -0.2647073980270639, -0.4905707268254313, 0.3168742779501993, -0.35429399736188233, -0.17340698583584696, -0.6979039408333445, -0.8003995662627345, -0.23716755491668307, -0.26966996826224127, -0.5045742704264182, -0.7262487802984885, -0.5505536934441684, -1.7421483789793457, -0.9477287248826006, -1.590129169262807, -2.009902562530003, -1.2348228868526796, -1.5606267610338507, -1.97037242820555],
[-0.4516111434162585, -0.021844964414661633, -0.4178387234702717, -4.8551428798453176e-05, 0.022876388433543997, 0.29337482952157523, 0.22081662485101045, -0.1616622078796856, -0.022463490696912954, -0.5047279102237843, 0.10772289484953196, -0.02149189177088008, 0.04068319948374202, 0.41144461579159525, 0.08854108435808447, 0.2821546695770571, -0.10932292519301816, -0.08867989369598377, -0.01876384451901837, -0.1829531152625271, -0.2274766576521198, -0.07691540838398211, -0.49500555670230273, -0.13803496757806466, -0.4124845124217273, -0.31522930761756374, -0.43091703689637256, -0.17718247727039227, 0.18883427849331155, -0.42148381613902897, 0.2522466320183296, -0.5384852879320406, -0.5281673037641513, -0.9706150516994149, -0.7810398511568912, 0.940965936110365, -0.8413752668603536, -0.2602855956003419, -2.0696183141644244, -1.292909148199027, -1.3509006306423748, -1.4121414989269534],
[0.29756793454354, -0.44141143536893424, -0.12489475851643989, 0.43703454287682086, 0.18928462845013175, 0.6650994600709649, -0.12048811508148703, 0.16010382617372587, -0.18840197367711778, -0.47651516601267996, -0.05601247282905125, 0.3716999826457987, 0.02142971114496921, -0.20645838167096742, 0.18571181277785362, -0.7272236451558655, -0.6925706259398543, 0.045646458192914297, 0.01450897019651745, -0.2166920103617781, -0.036024189690194344, -0.511241382214639, -0.7372513299782216, -0.2148666035494555, -0.1320400602917843, -0.6007833684957068, -0.6156917161729308, 0.03421605723667298, -0.482649939610956, -0.6747200081735967, -0.48206292971651254, -0.13104359956753597, -0.6652735174852293, -0.7382682698984421, -0.8427251156292305, -2.6429258592987375, -2.2589654893034408, -2.3607572707457156, -2.84053117165624, -2.5254374535098827, -2.389536729695756, -2.5113238633651114],
[-0.01112127503815587, -0.20352153894404063, -0.07152171414735457, -0.039328184248280974, -0.2019967168705688, -0.23394706052623612, -0.16684884726403668, -0.11414177221700744, -0.9705784574150373, -0.2070426231676081, -0.15265383310523545, -0.13473215717120823, -0.18616031289576768, -0.28426118807673006, 0.10661259941936387, 1.0771955474466512, -0.2740688997414359, -0.37266404066262104, -0.2911718036330766, -0.22347558224386901, -0.06688438675184868, -0.39400020049309503, 0.30980038970898993, -0.4197522010947602, -1.088761887058689, -0.48980820988517604, -0.4947636370019016, -0.2560571295799335, -0.7040580425019652, -0.24452079365149249, -0.7572344944690349, 0.30518756337064207, -0.742440463306243, -0.6861562527072547, -0.5636514191712313, -1.1483873215350007, -0.5762478513947407, -0.9842705497512787, -0.6917047987950115, -1.7220805422404275, -1.7639705927266982, -1.8749843332032616],
[0.03144647929518221, -0.06834472930461004, -0.025213387987018353, -0.08085725996566911, -0.02993776450500165, -0.11936163139830207, -0.026448298639266794, -0.08920778747754789, 0.07427692181540853, -0.15317101959080853, -0.1082206525540505, -0.13708138847819534, -0.12163297800605341, -0.1343232071016975, -0.20722698477992568, -0.47494328161276195, 0.02159238256428505, -0.45474106430778444, -0.26583493952867454, -0.5517754204567072, -0.1810747921573276, -0.12392758528452229, 0.09812521345240638, -0.23451804766627637, -0.689233517147709, -0.010214549174657462, 0.12691065881332808, -0.16526380592141185, -0.47454712167033974, -0.6430185453468399, -0.3567481759769854, 0.004784301376602445, -0.5697238585546424, -0.588418289974781, -0.3071004375671718, -0.8315228640941225, -0.532046982460897, -0.3347344588552225, -1.929041001177848, -0.6619026034029649, -0.21723819687680687, -1.1777844976973328],
[-0.31037393750396103, 1.0631263706135285, 2.1129520815062404, 0.3246799771712618, 1.1119029882946156, 1.1917864991881018, -1.5351976091338477, -0.4448862747670516, 0.3457040557320539, 0.575601893783132, 0.48491262979617533, 0.2847307399334848, 0.6157004216155743, -0.7669332601649088, -0.34544389233004896, 0.42984012692306045, 0.3402097023888919, -0.018805628559763757, 0.2011440733302975, 0.4443434388985859, -0.3406642211333884, -0.19447387812280376, 0.16019671563613364, 0.0072428225736924385, 0.19930957901795843, -0.4689196541106822, -0.520713156568054, 0.12912691147175218, -0.6164864852806304, 0.2842896355150656, -0.9440894707278605, -0.606740475340588, -1.1163642345211942, -0.9769285669150772, -0.6589791572840773, -2.8655261504995595, -2.5363739929418885, -3.2647894497458188, -3.3196477508162485, -3.240652939688674, -2.858741778125113, -3.135056261366421],
[-0.05053508248514265, -0.01419786293084266, -0.14568372947304625, 0.01901623338140011, -0.2045744793204934, 0.0949492032476894, -0.06505271093867034, -0.0503483780922286, -0.05891715179599649, -0.18265098684565917, 0.0327106908739856, -0.0855525442722646, 0.08377613541324248, -0.1475926576772878, -0.3038527897694679, -0.2584202964854224, -0.2117877650790714, -0.21124066166524605, -0.17738295267938017, -0.2212322708609183, -0.22717515250111087, -0.2754089438732761, -0.3778202656962322, -0.15661369366463593, -0.3485468657602185, -0.35896729365619656, -0.11552282842916341, -0.047823304136677104, -0.6691056278008786, -0.49777082783581755, -0.4012461757591947, -0.232109707975473, -0.285564670413969, -0.5705015120834932, -0.5009773670370627, -1.5798445383792092, -1.3045950798095702, -1.3705163007166559, -2.024010217135923, -1.3823134965184414, -1.1481545686614294, -1.4383762327830294],
[0.1825931085597441, -0.3315155420123455, 0.03265011378682041, -0.13495812978891716, 0.11406249105749143, -0.2156494534632547, -0.0713489427722153, 0.08402822615785774, -0.17027555073244782, -0.0978218692102602, -0.2846712955068164, -0.15050868812252596, -0.2648814688815325, -0.2151323538030211, -0.007463842852541993, -0.46299918769706744, -0.27620646216255823, -0.4163666081625524, 0.3240527060811471, -0.4418942774452318, -0.10069134833008636, -0.16595380942553262, -0.3394682895211578, 0.11407003184669921, -0.3735183829358835, 0.03451382474455683, -0.35420847810564793, -0.3849137775215737, -0.3676103566120294, -0.6267577975679539, -0.5812275933623621, -0.42151194813508014, -0.4011879088011997, -0.4987156529090158, -0.598124966769446, -1.9732571423208018, -1.52040730246756, -1.8603169113640212, -2.035591823912722, -1.7000825460308877, -1.5347839795213372, -1.954965305984288],
[-1.527606906396132, -0.39041011235571477, -0.38723000620974546, 3.9801335868572885, 0.6014348393921989, 1.2507152930580052, 0.046708905765787616, -0.4654684517559853, -0.6605340952920385, 0.21801272421224888, 1.2505931487670825, 0.1701765895805803, 0.521492145523336, 0.09894678439040396, -1.5685340532712344, -0.9013149520078924, 0.2695094331547238, -0.6308601232567989, 0.12656618094415512, 0.0741515478791662, 0.5336602063419377, 0.7120290268998815, -0.3949159218015413, -0.1742344665573241, -0.5840233827060634, 0.03529967515287228, 0.5892712423632631, -0.9246721136020175, 0.9645788594223674, 0.23830991567888232, -0.14536798523480662, -1.2910161813732233, -0.5798246887123968, -0.9955978156772135, -0.32338918361006364, -1.873507843222537, -1.8362104448724978, -2.330028612664706, -3.1086122316538236, -2.708851169350101, -2.430299266389264, -2.7799652019044307],
[1.3109768506115635, 1.3366133818876578, 0.736702979504321, 1.8457601935168038, -3.8564620201581707, 2.378064042215934, 1.4278698898891602, 2.0145758045393127, 0.028891617739506367, 0.6461032259734298, 1.4016236398233328, -2.071521203809601, 0.6513530100902666, -0.09433633853105863, 1.720943350078073, 0.12018378329441569, 0.9760353108891849, -0.6867445565107881, -1.4288504049547572, 1.0112534233083976, 2.4526128173711386, 0.7018584630173474, -0.6504866574312788, 0.3671691050095679, -0.02548002013586239, -1.6702295348002618, -0.7118044848459567, 1.2937673440367052, -0.12064576088445349, -1.1849325428408002, 0.05115507924707429, -1.5366620332298073, 1.6418052559956808, -0.9146686722637658, 1.0560706578338515, -1.5777095499380338, -2.438832717949051, -1.2641289890822252, -2.466203475097924, -0.970531631690902, -3.1925327321435057, -0.30784603528678683],
[-3.6558401082982126, 11.121672104056495, -1.8485878619915879, -0.3650721325250142, 5.176132032534697, 2.636668574517547, -3.7824696173464343, -0.6459036957760879, 3.3672771538312167, -0.8153158686221698, 2.1850762307223777, 1.975971271783044, 0.7391834358955235, 0.3501946270683219, -2.393825252243056, -0.01867411474173345, 0.5849899508812862, -0.990744317896938, 0.16907093573493617, 0.9196986275357146, -1.2799447132339683, 0.8557969858638064, -0.7934740215565408, 1.8615048506150378, 0.18746523547219487, -1.9321827743629851, -1.0733914816605372, 1.75040850025125, 1.2622941703213533, -4.340327627185131, 0.5682238353961709, -3.0849226026377057, -2.3401012295458465, -2.28800697146899, 0.2340104270191298, 1.6015807075212027, -5.144426578123614, -0.5013789251063825, -1.4836847094784544, -1.8202986375560253, -1.748866735814522, -2.287514591903213],
[-0.3731904571163489, -3.403785218920649, 4.80792544926141, 0.7145881465937363, -1.5584628530022033, -1.4082395716877338, 0.09199328258304666, 0.2257983427809623, -2.441264652525846, 1.458616160544523, 1.6741535633734181, 0.6505647070727919, -0.38619813780481055, 0.268975530582577, 0.7116411025081123, 0.3012859804922275, -1.6272679272419628, -0.07694176735184743, -0.5126544817508502, -1.0979385247917697, -0.17357213008026234, -0.07130517468977941, 2.638269597059723, -2.267165551989435, -0.0823740797568893, -0.7624037013808833, -0.5374082649326007, -1.0476311641645664, -1.6345783035284671, 1.705252306421002, -3.5658680179378743, 1.5821489598596299, 0.05015355558412718, 0.7076631561568285, -0.8397909162261276, -2.672741185498761, -1.1513527757520323, -5.585620569421987, -3.7366267162032565, -2.1296286621224767, -2.7640178447125616, -1.8358044979364747],
[0.4212830735712897, -0.14881158543090672, 0.5168742786809842, -0.45682818125965774, 0.13576995484032245, 0.365756321876287, -0.5795005456641562, 0.13130013081778877, -0.5790819636359623, 0.0734384903634512, -0.6513359977771822, -0.15491293400497644, 0.7658134459287275, -0.6825022866720929, 0.22696090992557488, -0.36895012618531514, -0.2575570721311903, -0.7436944108503064, -0.6105353969015485, 0.2223745725885351, -0.10822994225965839, 0.10938747334012723, -0.06912940785276835, 0.14377982417998694, -0.35769696808722096, -0.7153624359715067, 0.21520854631945827, 0.21076192337924415, -0.9043372153476453, -0.3914761520392313, -0.4593973592899839, -1.0397977555601559, -1.041926973219723, -0.26990942223018166, -0.4076358487194351, -2.5555957963766014, -2.244616422646113, -2.4709421119792947, -2.650643419726177, -2.53942103853656, -2.4736536708823924, -2.4797985685075643],
[-0.047723113508650515, 0.07241777624404437, -0.14561863718390328, -0.3380020565917887, -0.05767190739502066, 0.14292320735275044, -0.024523158020438595, -0.186314924700179, -0.08945962263172137, -0.00719010384047605, -0.024902004432975383, -0.19048001388888264, 0.19319721971754558, -0.20082185168935035, -0.22668510516047743, -0.38182131955901866, -0.37401861366979106, -0.12133283302363679, -0.34389823566877215, -0.11316388796825932, -0.16057259129156273, 0.018230252146257914, -0.4453472804093933, -0.5902025243306521, -0.5974858716196189, -0.7355380484860323, -0.2523744511959463, -0.004476559040712642, -0.47950069659344174, -0.4514017774743329, -0.4631304193684104, -0.28250944584139176, -0.2504173532977062, -0.43948442917332015, -0.4866353430926342, -1.4747187873828274, -1.3300023576846272, -1.3179024611271415, -1.3862287822585047, -1.1175589283375569, -1.0747990149553956, -0.9918481546739063],
[0.6045250172047014, 0.4117878202616513, -0.8540019961286859, -0.6223839713837883, -0.15879381584894345, 0.13123179396599294, -0.06730924808144377, 0.27190771257982643, -0.06915565705047108, -0.6626796176593841, -0.3925994663301831, -0.259400653942046, 0.31768373823415014, -0.30129130390784287, -0.13964466519661126, -0.17223566920833402, -0.19611177348372985, -0.5157027324950272, -0.08571166502006476, 0.04461368582286601, -0.3457000590984071, -0.31460700220127874, -0.4052313309657127, -0.6688207579162869, -0.07979415711347568, -0.19798851104133472, 0.32053044664885877, -0.25205542583101076, -0.7648758672931519, -0.9516501010396036, -0.2007836686923709, -0.18216918370909715, -0.3538589446098576, -0.5290319124536605, -0.71501482605826, -2.2695670135785666, -2.0255389111787157, -2.1261361972713386, -2.28550220212355, -2.277273186036577, -1.830599633710499, -1.9517456535437172],
[0.5561118798043226, 0.8562202102520102, -0.8226885467351612, -1.7903980995077218, -0.1264425789136824, 0.2315900930461192, -1.318037699165097, -0.9345932883114173, -0.6374382177861827, 0.3534919801865069, -1.6461921544640874, -0.6058983601713923, 0.5628355947622791, -3.983308817241939, 0.2778537680008432, -2.4701411861401255, -1.7772201061888881, -0.25853140920480855, -1.5670532413199922, 1.566888347175815, -2.1137744675880747, -1.481041719322257, -0.3259352362878626, -0.3764505931690228, 0.07050917467652025, -2.010776983356286, 3.969963589337386, 1.480642279908515, -0.8372370514975734, 0.9934602807679156, -0.5896849188341203, 0.7632237004784928, -3.0553779823347242, 2.8363581410403875, -1.3907535226248773, -1.1305122170843664, -2.112875434695618, -3.057011487619992, -2.1478473862798673, -3.6230838017935194, -0.9337486130299717, -0.4778427597977103],
[-0.29256117950437105, -0.5810181023849507, 0.864887954538538, -0.18084433342139097, -0.08697688328721002, 0.3834733641562775, 0.343631371596226, -0.5425798445931574, -0.5634366803944919, -0.42603392454265665, 1.9061525056324984, -0.5317696029012735, -0.08945907278944783, 0.5264249543969532, -0.25148214625157506, -0.4356587950123555, 0.23050003248369488, -0.006570343780669767, -0.04305827638603564, -0.3422719102653929, 0.24698915303364682, -0.6984123557929038, 1.388756230724511, -0.04961775301712691, 0.24325457204532566, -0.716797379339168, -0.5791331366133404, 0.02275891329737502, -0.21197291189243625, 2.38782638897633, -0.8699664943902495, -1.63678094982754, -0.10319711163722659, -1.099753878918518, -0.5988788773077978, -1.5105707026338804, 0.8287420741356452, -2.2450979965277456, -2.919423547983808, -3.024480719284279, -1.8222867629900399, -1.0837514330386067],
[1.190684990236935, 0.8968114731014096, -0.39759017491050663, 0.8176149019133476, -0.5477310806890807, 0.2708089369009753, 0.5729835694604613, 0.6398047937565697, 0.4330302453804394, -0.5569993724845669, 0.5096736049040858, -0.552828698691963, -0.22247161356521927, 0.5064809588748969, 0.21153587886936998, 0.0034832431452271355, -0.10852210383952972, 0.6048444010006827, -0.4819166109248479, -0.44137627660953543, -0.041573325014977985, -0.3704913845329974, -0.46360830470425096, -0.5079867248058872, -0.7290188948912684, -0.16077849541240413, -0.44090683223702887, 0.08341794964968578, -0.6652393339010751, -1.0366175495443273, -0.34252479155654686, -0.2673166727290777, 0.6482587437280036, -0.5615531594335809, -0.3768051207070117, -1.8722493316074063, -1.6100164356589317, -1.8030734964210682, -0.6492769477425483, -1.3030976473534719, -1.4697293366684967, -0.559375025385892],
[-0.031334009307891014, -0.08422394020558471, -0.02861352816177834, 0.006823266084860057, 0.02486509349122966, -0.11387379183545113, 0.025405684974626298, -0.03774496562030545, -0.17754210276606183, 0.022703253511149272, -0.2591156371489086, 0.02117874705747783, -0.16286167884471783, -0.09130558655804603, -0.16403353204278406, -0.3561724062946038, 0.1665210612010843, 0.006423954152860457, -0.3718023960428928, -0.3822588116259174, -0.23767980732554866, -0.19820702232844511, -0.18261820675305307, 0.10875186269451777, -0.17238383706325183, -0.15933825187043035, 0.004980352513003093, 0.046842996465745135, -0.5211639451815027, -0.015743688681577998, -0.11630738674043338, -0.340567562132799, -0.32813735015058887, 0.06982568246134735, -0.0981450924024571, -0.27763284766655116, -0.05974971620981051, 0.19227572998895315, -0.9506684033969005, -0.5362305571418439, 0.1290640075493577, -0.31141852600788894],
[0.21174875547895083, 0.017602807720730584, -0.17986869290784627, -0.21052563446067418, -0.258015855852489, -0.06575574209417834, -0.1392973776435863, 0.13058023745795588, -0.07099549292231308, 0.02771366699721861, -0.1191638329387721, -0.3974118990265639, -0.17527272895745774, -0.2726370422316531, -0.03998853421835949, -0.22891315289730804, -0.15860144304873836, -0.11332043122955608, -0.2621006405343554, -0.38242112258669275, -0.42367940947273036, -0.2439642360949186, -0.34251081925101506, -0.3708128213032933, -0.3583507164798782, -0.6513219751271807, -0.11870171685573107, -0.05688203584557555, -0.8329675375653612, -0.6717253741093143, -0.4316417104006481, -0.15357052410359515, -0.5234727134540519, -0.2937625737711696, -0.45049613008471145, -1.7798995519602192, -1.5441437531652142, -1.75702450669325, -1.8722287782690323, -1.9330916785294832, -1.4756657047560668, -1.6376558925069393],
[-0.0890442818991385, 0.12272794752169396, -0.1935027521558569, -0.15083640568743506, -0.17553538506271815, -0.009130611903613447, -0.1186653350749734, -0.2349832887879201, 0.04587966915990948, 0.028591538848643135, -0.17066356872919689, -0.0035589529296828843, -0.045636112734793836, -0.11895491643719265, -0.3215511838356355, -0.2758798796044619, -0.32644523343513954, -0.3198109073810296, -0.2407488396566647, -0.265818157237954, -0.27325385817046044, -0.2023127129982758, -0.41371545360558126, -0.42711652495344943, -0.40376207637202705, -0.4710825644243404, -0.3717709866433864, -0.09883642064929887, -0.37209539054984536, -0.39613815023081245, -0.3879191361054959, -0.20313356290603604, -0.33369933653633943, -0.4246306960757423, -0.457374579173498, -1.1141912217228243, -1.0774725196889632, -0.674029434199846, -1.2405370038520593, -0.7998286465874634, -0.7816682320835397, -0.9900697595736904],
[-0.09311921309509215, -0.2629215202342256, -0.1370724111282271, -0.3409136678896097, 0.31921434573550145, 0.17330354278082472, 0.022733869074230275, 0.016915192730493706, -0.29698884421975524, -0.22999526460860414, -0.2540401537158546, 0.13103471163317518, 0.12501798198121147, -0.24196142051035452, -0.13184973837572367, -0.25978336819810927, -0.3423791095013798, -0.3735480648039333, -0.03484869484272121, -0.028280350442078325, -0.1726706441167837, -0.2866508162173614, -0.5049080941911915, -0.5112669489320512, -0.5547096047971921, 0.08506111769406216, -0.27610413592688593, -0.20394143436325335, -0.021235489026902032, -0.5498930039674701, -0.21922840207410657, -0.2365532685850706, 0.01158597700791373, -0.43219495441205014, 0.20043721186301594, -0.6082747696632339, -0.11022797965391884, -0.6838698653195872, -1.1217705891701213, 0.03225128164452439, -0.4067739072665635, -0.3560292494310238],
[0.6798964937944776, -0.002233627233861687, -0.3007324115763478, 0.14908580318497683, -0.9084577165986091, 0.34119323560494147, -0.46345418313105946, -0.44217467148099954, -0.6824140234655179, -0.5441425239496628, 1.624950982663372, 0.2373122051373233, 1.7227419292476218, -4.1895250683311245, -0.13428015316740027, -0.8666331962187162, -0.9003397795349264, 1.1197531687820623, -2.1467629546699944, 1.875703122945023, 1.2544423444783557, -1.9551476587917072, -1.4228665479404436, -1.134704843360047, 2.84060708796354, 0.8041546161765175, 1.1747506535089676, -1.444182090065313, -0.23267403433057712, -1.3626425626059515, -1.5129872789840255, -2.217332381167728, -1.97246422733763, -2.4639481229921443, -2.187730555752499, -1.9755982029846266, -3.995459125774435, -4.578952802087102, -2.263281442210975, 0.03922779097502346, -4.01151659500433, 1.0247985266517021],
[0.4124876614362255, -1.144466767422251, -0.6230029468369158, -1.315070672139193, -0.47249746153693223, 0.2059181269908579, 0.3556464349788612, 0.2221703289904552, -1.3151686667766473, -0.6918596381871687, -3.9834429994164764, -1.1156363277167642, 0.2144716461538189, -0.27141089068554647, -0.6433630875677012, -1.590512967850867, 2.5933208317222887, 1.6677939798591828, -1.973992714386354, -0.3934671835755736, -0.5747525876890388, -0.48599348569552103, -2.6548771817519246, 0.18536984253650712, -0.14697859027830915, -0.4091824696272054, 0.27239039897711387, -1.831275393813116, -1.1247584444385579, -2.0103611194046063, 3.366837537859616, -0.4334938655830044, 1.4281711609787526, -1.892622370636005, 0.2350628363988976, -0.9433574331314245, 0.6357448211598578, -1.7406798231135425, -1.9939501370845998, -3.483786848531534, -2.642656068289048, -1.1195822616118067],
[-0.15974292012711622, 0.007098340011791697, -0.1293361791160403, -0.32106191755896674, 0.028051262446760147, -0.11172283519804894, 0.02538717426019432, -0.09357448568162109, -0.1355670682817703, -0.2405081472436626, -0.5847097924507726, 0.013448271262784908, -0.24043480910802795, 0.0985567796646826, -0.16650671555441665, -0.189772999306855, 0.040829322915790146, -0.45005860106203865, -0.19416712423191118, -0.33903202905385293, -0.42785582035603836, -0.20292999279950832, -0.21745287935348437, -0.42052462385664313, -0.5905221673724579, 0.0005017592609291015, -0.48496742092407075, -0.10672371427962545, -0.7663436188887579, -0.7791898554760016, -0.42514993430829806, -0.022635493613941987, -0.4033485448910758, -0.6496700561840313, -0.3747899890487753, -1.9685158354495076, -1.3977998568260868, -1.7148957421635302, -1.9786092409179898, -1.5198328558554262, -1.40948278854579, -1.748377533980267],
[-0.36665148163886235, -0.0316324471697894, 0.14763884522647086, -0.1669131843524509, -0.04222099285239897, -0.08044925739682401, -0.04204787341497238, -0.2364919006793915, 0.01275987008652857, 0.15011677209291446, 0.06276969136504844, -0.03072153002022568, 0.04309666692311944, -0.21799218225117445, -0.2412936156108071, 0.13659602604196672, 0.24344954201834876, 0.1300296206886888, 0.032250749410441204, -0.01115779663106605, -0.13355948699683995, -0.2140863434001557, -0.11630171343138739, -0.09663325757113002, -0.0984363057874446, -0.42380478390750514, -0.32441756021489304, -0.12539817930684855, -0.8566129046835075, -0.42991785548235134, -0.5413937015798599, -0.15194313584374203, -0.5211666654459849, -0.46997410840066156, -0.9419309931272393, -3.2265712477723016, -2.914682803565263, -3.3330602058910013, -3.526827414290847, -3.305652562176362, -3.0324885654297615, -3.2140757463338674],
[0.12462663136249305, 0.12430763156669825, 0.08282858676556878, 0.10308673424988422, 0.140870385967903, -0.13503081187793023, -0.12618882921844965, -0.010707867580726933, -0.11964694819413148, -0.21277635636686254, -0.12395973132276138, -0.1475271558805604, -0.3744619025279689, 0.03807781500959734, -0.1328243372729125, -0.14198072969798417, -0.3515083173467349, -0.02503377625326441, -0.3585714916734906, -0.2571179669454799, 0.041306473056881815, -0.33946416532904267, -0.2645871876431484, -0.21242826308501817, -0.3494990204559044, -0.47457039595700634, -0.372162486021284, -0.3453321297797882, -0.7432934429850308, -0.5091329541694153, -0.6235518327280891, -0.156133374365804, -0.44134875475368485, -0.5950585904635037, -0.5963320742225956, -2.0888028011110613, -1.8923217136905062, -2.0352188616386133, -2.5107170201450595, -2.043432948768078, -1.774627333656393, -2.073085351115507],
[-0.048542379536841936, -0.34291679811205783, -0.025041272323646706, -0.27792651323316747, 0.011082482627695489, -0.06292737577446807, -0.029284257347773946, -0.11237789165765198, -0.11084809854367576, -0.019664711664221766, -0.14421262152225642, -0.04740361039788097, 0.05113879132412878, 0.0441682015715918, 0.0623409063680223, -0.20811277490801247, 0.019254391213243394, -0.30844267439461326, -0.35757105082149143, -0.3020200256575922, -0.256579061543816, -0.16480196770773167, -0.2713546817372426, -0.01844092859551007, -0.2243209002471551, -0.5150115339913558, -0.4447580782437222, -0.16595594080054463, -0.7002101579474135, -0.6273017071534298, -0.3944937743520489, -0.20716286718348356, -0.5319918440603133, -0.6580742575881847, -0.5168769950995946, -1.8948003820223944, -1.6033527588936614, -1.897400635474442, -2.1755821640244433, -1.9479827195230632, -1.539770057924445, -2.167061344903691],
[-0.12031385248501125, 0.02792359999473707, -0.24793197601937117, -0.004078952574846141, -0.26609642660537386, -0.022585895095143683, -0.016289079274892513, -0.17792724753799602, -0.2121293825031089, -0.009813360117278985, -0.21169103328196365, -0.2249240922931987, -0.28965784259389593, 0.0041089651507809286, -0.22322130658576433, -0.08231627264330352, -0.1424333705652053, -0.5302475402615267, -0.20340764130490036, -0.47183341376635535, -0.3177442236815622, -0.3056656354954733, -0.15785756755161898, -0.4715320538548597, -0.28331612798638434, -0.5024699460504511, -0.5377291803873426, -0.06798828488264347, -0.20555268562250578, -0.6875760655020436, -0.3282266376445247, -0.020575940945205626, -0.44940378584515805, -0.8307182494010449, -0.4619634644505696, -1.0415430243137287, -0.9392583991600036, -1.159752942237566, -1.6690789221426514, -1.2424050811760692, -0.9113951908133772, -1.3396399836767905],
[-0.1517117952019397, -0.08876670538560572, 0.012150103340415839, -0.25154753482278125, 0.12305340796342948, 0.003756674316768769, -0.13621080548718018, -0.09915890779480567, -0.20768748164954468, -0.2534665193730988, -0.2009735817895099, -0.1326573708981096, 0.07897224150586554, -0.03264809620284406, -0.1516990752533594, -0.26941740257808394, -0.05120995952836276, -0.37035180317587757, -0.24674422758409642, -0.48917157284855534, -0.2295456109849622, -0.31382132020157427, -0.1380456202545731, -0.1545104123153531, -0.4803266300951863, -0.23600332709246077, 0.20559649068238092, -0.0932287453333365, -0.6375165947718465, -0.3279911695294224, -0.45924873950181205, -0.3077280045061227, -0.5818848210237729, -0.3279245515773147, -0.7152200054304776, -1.0399924705457588, -1.0596702937066846, -0.569248182397246, -2.2463925294577463, -1.3180877664923476, -0.7506392487575982, -1.3717087553827023],
[-0.004221440566216553, -0.04304850618617948, 0.12572939519853477, 0.04402133126790379, -0.006984747812437168, -0.046527709000303265, -0.06388774170301702, -0.08597457444256187, -0.1718556872666245, -0.017893131588963537, -0.14831993686556508, 0.037727953937117385, -0.06715872036851166, 0.011763636057291117, -0.19922360901929767, -0.24308978875903262, -0.02095525602313516, -0.2936558692890608, -0.06263019803760111, -0.21956452389303888, -0.23602013552383863, -0.3398308087050425, -0.10138231533412305, -0.22020044909497374, -0.3562492590708695, -0.1782711104881712, -0.4115854407214311, -0.13895496409353683, -0.6061367667834207, -0.21255153666203508, -0.5642904710636049, -0.38188905111134036, -0.2750293817006718, -0.42165092437013796, -0.12754673984153037, -1.2834931556886633, -0.6991842809004231, -1.236162763570268, -2.0138770322848045, -0.742511166635311, -1.1159794649289734, -1.066796246960134],
[-0.005261252448566998, -0.42519718035214676, 0.06094743936208573, -0.24182530740332178, 0.3672750141842767, 0.11275607593647173, 0.06721063058990775, -0.2069085470756323, -0.33931993611292205, 0.14758986378014893, -0.4370151325359101, 0.16373835980020568, -0.14121150233473054, 0.05316127932402193, 0.04966512256031663, -0.403158128651399, -0.11547300652109094, -0.3266662985813667, -0.15804666718658134, -0.1913752562886297, -0.1076320343347427, -0.30443102450105225, -0.020551852170123622, -0.18606886715803164, -0.5003872430741896, -0.1764382268253867, -0.3469820888244408, -0.26759890734362857, -0.540653680974199, -0.03675188278425622, -0.4262283336957074, -0.4813861518717875, -0.39046573899537945, -0.27846409985843695, -0.6910807156292309, -0.9745885926130592, -0.6536679231788667, -1.0044561409682298, -2.008107873763588, -1.1167720732205058, -0.7479341824826055, -1.308802110216948],
[0.3817649348302674, 0.12117761723446323, 0.02066569994395979, 0.35589839155980013, -0.3599479367511083, 0.020126261525055776, 0.041054273196772795, 0.335695675262668, -0.1462528922945298, -0.24523015790448224, -0.13639686690893058, -0.08191549300955621, -0.23920107340427366, 0.01319974486622617, 0.05296774267045481, -0.13917350598523864, -0.2197510707162671, 0.21448306500019557, -0.39847449890644154, -0.035092610100043796, -0.39945800421588784, -0.17800871017426245, -0.5293839582152008, -0.1865696763615842, 0.0036041034056549385, -0.46317025306942694, -0.3193371623930532, -0.1649605668039669, -0.7722766950712688, -0.5967259382829612, -0.2524649622558386, -0.6230307741591107, -0.13319231833165868, -0.46197244251014, -0.648261335599325, -2.0377546037008307, -1.5183169565757884, -1.8499635567349908, -1.1436068985221552, -1.877346568206476, -1.693177529699047, -0.7681743806943804],
[0.21752479222226567, 0.18947451092000123, 0.13473734208266358, 0.034714447104576995, -0.1132386904422314, -0.2348927918404904, -0.0016771003498017645, -0.1985335102246962, -0.32625422052062825, 0.14391387773845934, 0.06509086050709133, -0.1333299149498883, -0.6424327332863728, 0.22186711373113088, -0.3929096520496408, -0.19189755849235351, 0.0009889719434971508, -0.05319106554914018, -0.03597725238912957, -0.2271183061859901, 0.15810038086942665, -0.5127893341842957, -0.6268320159099535, -0.17537273843374449, -0.458123216096959, -0.2989409588842297, -0.5260079668625062, -0.25799285240500874, -0.784616714216146, -0.6332964729050543, -0.45390714344746746, -0.24795863160370005, -0.3890546395888659, -0.017350038202862555, -0.36722529610658866, -2.2472931823364375, -1.530472185444158, -1.6092400028903315, -1.9928780352559639, -1.4249022914748783, -1.1232151441837088, -1.6943975959536428],
[-0.22734624349406043, -0.14892937643074106, -0.13127746951548425, -0.020289299008439104, 0.13645607918810365, 0.1449554230382, 0.19047614939581844, -0.2716610132872714, -0.23260245136817928, -0.2973005949104195, -0.16143133223154615, -0.14448075505132918, 0.0907826700816539, 0.03032819170042864, -0.2840731237396651, -0.14497542753597734, -0.0022776473909254195, -0.10948539543106985, -0.2083419970856859, -0.2858968575857427, 0.01865631857250285, -0.16065942916986595, 0.12211009901452588, -0.27587204072499283, -0.13536399603561192, -0.28465961763527864, -0.27567603125974777, -0.2391536143402236, -0.6979494834116402, 0.02608806745003632, -0.5128303571138035, -0.34256248099451514, -0.5250188836488047, -0.4950960471367579, -0.755711823473937, -1.5551876551376425, -1.5898068765287034, -1.994816813473923, -2.4522531381200987, -2.2394706290586246, -1.8211102114708324, -2.140679596172768],
[1.2532515771861323, 2.3594950455376345, 0.2087536986352691, -5.241537979149789, -0.020027871110594588, 0.7414603385286448, 1.7098491463472623, -0.3290351964714401, 0.4667469068666911, 1.8053982860932625, -1.7787792847603776, -1.287545818695623, 1.0454260304064777, -0.3713113919699225, -1.3931973312998798, -0.6299796043496824, 0.219203930425553, -4.916822122424185, -1.1094212323299375, 4.994396218375029, -2.167353083232213, -0.6762699498488313, -1.0096599143151916, 0.6343182534226185, -0.005128726038114811, -1.7125644538649971, -1.7731525061219668, 2.1194332549733255, -0.8621904224989722, -1.7210178997162424, 1.7420460048697548, 2.025899726880549, -1.7580412815247344, 0.7035270309173293, 0.9162143945972693, -2.3924725663891175, -3.0748879744878783, -0.07373247208482127, -2.5015336222851237, -2.705391703261283, -3.6996781145749256, 1.408936232093259],
[0.058860339440916745, -0.1076377653557524, -0.14295382315710212, -0.37641981282946535, -0.06891630577159714, -0.06009023722412798, -0.23378115889894338, -0.09908710809699352, -0.2966424065411783, -0.2038694970814494, -0.362996018767265, -0.18838238804605323, -0.0440175530340475, -0.36177440088715695, -0.0470276474490146, -0.3240983758784872, 0.3308299616799693, -0.4787223739158542, -0.11846664172314886, -0.3853943548266603, 0.3658944052764844, -0.17807630906512423, -0.22985032927301693, 0.10035906103696739, -0.4035153042815941, 0.4916039715392704, -0.15716561464312837, -0.7394441857916628, -0.5394645818753232, -0.7467942611043366, -0.4059420996199608, -0.6759174392327731, -0.2101475592252311, -0.3319153207895752, -0.12071531614683761, -0.9473005218507287, -0.3273437643306178, -1.5922399526860305, -1.8828419204517837, 0.6028706840281175, -1.4069947842366286, -1.3921873971356125],
[-0.019727819378433206, -0.5938424210812312, 0.09807301567620302, 0.2660716275599459, 0.13454252387013313, 0.17691282667126698, 0.4079929278904269, 0.00929018772839517, -0.5618903650939794, -0.16326998876101345, 0.04900370823498695, 0.045928110828970116, -0.11331907763414158, 0.3170100667164974, 0.0069215356954751665, -0.04943544049286197, -0.22385849046266892, -0.011441851889649777, 0.01633064148235381, -0.17425249132922246, 0.04844510926589933, -0.03393105817296609, -0.22410863076425158, -0.04154862168148621, -0.20496855593889612, -0.1576214311585251, -0.3220285110291099, -0.347613797345803, 0.13134628351199798, -0.29173585255889395, 0.11559263473249172, -0.370843021099199, -0.6115532442841696, -0.5237674949919766, -0.7176106215512927, -0.7910583769932291, -0.7538024103471858, -0.8758176870034091, -0.9066508194603784, -1.2845315872314522, -1.282723325921669, -1.7016645756372457],
[-0.11470728101730897, -0.35362504772670966, -0.09217357364265422, -0.5874257799084411, 0.13421003115630897, 0.3369011793639957, -0.06557395048451432, -0.2719472257180659, -0.20763468920214098, -0.28687420516387735, -0.4354717382175891, 0.16269508648181485, 0.26123943508759795, 0.18284867223567658, -0.01936903216450263, -0.40206487494208976, 0.05832992406413027, -0.5108920475698748, 0.21377482552497412, -0.1871830746474656, 0.023185945280758086, -0.19913837092546158, 0.19770483667663988, -0.06285509953709299, -0.12016945976451317, 0.01883290505303511, -0.19571888430461606, -0.14552286155862945, -0.2885587890398185, -0.29934335011205776, 0.03471487466700511, 0.6665731421895262, -0.3174939612073475, -0.706599501326991, -0.3626145836000263, 0.13650936325132698, 0.2293811733469434, 0.1762398014022819, -2.2083752549026783, -0.9793146089107896, 0.08606824142896624, -0.47811913157722685],
[0.8157509790950958, 0.8246901377309052, 0.9760273609961088, -1.2797618904646264, -1.2014696747939977, 0.02725660153625966, -0.32136892688614876, 0.5276802404094206, 0.5477416004751039, 0.46708495087663676, -0.5817191325130875, -0.6435104976918296, -0.16190240514783782, -0.05021754225406848, 0.20031271834724707, 0.035700136114337734, -0.095286502602099, -0.6296723800623257, -0.8497097343946572, 0.13230012353174447, -0.21883242384618498, -0.5278385805749838, -0.612755335429185, -0.6661775079439248, -1.6602546071575295, -1.2617699733369647, -0.4589170423017514, 0.4737988332075954, -1.0256842016484622, -0.6812789057821336, -1.0858703441090929, 0.4768436480082213, -0.30888811291653273, -1.100340384733062, 0.14242468772185662, -3.3749832903458126, -2.8521765647448585, -2.9392991778702924, -3.4089729948072125, -3.017475519326098, -3.256638715844981, -2.8399100121998626],
[0.11621495612712333, 0.029256599559861766, 0.12554404497589733, -0.04612958287594163, -0.09613772941437383, -0.18246937263900662, 0.14875349358591705, -0.03159140438997045, -0.03526835601795762, -0.12423137755438547, -0.23371816229559766, -0.2749421947089281, 0.06956537827983249, -0.07913989372865317, -0.27762583472361907, -0.11803557166201963, -0.038633164887636205, -0.39453188356580965, -0.14376251256256992, -0.14375304899204766, -0.11968887850655403, -0.2228934869203999, -0.36263759839504894, -0.23781759164479524, -0.7067014273871659, -0.33443542271091264, -0.37489786239211115, -0.32305420213027725, -0.5301441440072955, -0.4130573534803521, -0.42141226242703206, -0.06745156701920414, -0.41180316305065856, -0.40054279682674726, -0.7277294362757873, -1.6199245462268124, -0.6978879641874218, -0.9288347557977615, -1.4840695556509464, -0.9174779670360815, -0.8640156537983165, -1.473396537003175],
[0.32278038396809755, -0.17010756383890796, -0.07999380796765077, 0.007221941010212006, 0.33861668711898374, -0.13347419852404654, -0.022736121366277087, 0.28616031728513575, -0.2052087508589545, -0.2194420724721743, 0.024324763888522737, 0.1088135810161974, -0.3590432918297202, 0.08179792837339182, 0.05494264990219738, -0.18788659517277784, -0.21981878946360775, -0.053592271727407374, -0.19455251797158252, -0.28465465824486574, -0.26332152186199154, -0.459088634834725, -0.6046723384438405, -0.3720498966465965, -0.40987435808248973, -0.2849769196137446, -0.5695779597064214, -0.15345050804273597, -0.6222331961828219, -0.29803473725376445, -0.14719788955990237, -0.6373548694344647, -0.4455922725241697, 0.1136775138657837, -0.46187877687683576, -1.9906354760925626, -1.3272862143348545, -1.4016021814942485, -0.9703183118469788, -1.3243718680096548, -1.4426840752396597, -1.4516564481029244],
[0.12660662287743948, -0.24462413893612486, -0.31053723616261397, 0.10461722093021819, 0.00954274025921148, 0.1819510466946534, -0.1306570223377324, -0.1742807964620335, -0.20715769947197918, -0.5855091803838688, -0.38260735951178554, -0.14828271078020325, -0.06961579926368523, -0.3081260021386004, -0.21116738835402216, -0.14928789611905316, -0.5075042885065671, -0.43744145927508754, 0.07777652242839629, -0.07693968612010146, -0.23722324086345953, -0.4608078339990089, -0.16920901554811435, -0.5185772303936038, -0.5062980426590479, -0.026684497144626392, -0.1659532479381362, -0.3480909181919746, -0.5168533751350012, -0.42643105667621295, -0.39766462947629583, -0.7356150658082917, -0.29419478976952573, -0.21597255336306237, -0.7373970372179283, -1.979120635612064, -1.4934295191136058, -1.7749455874787639, -2.1695079252762697, -2.0100374233909313, -1.4203502440872529, -2.215435196127283],
[0.20582774520324373, 0.027611499626923598, 0.248362129835096, 0.2823609238612, -0.029856092662217944, -0.13434104138298078, 0.16393159115879594, 0.1614436281760561, -0.008433362700439942, 0.050355115623645315, 0.1749443921269425, 0.06513979984345736, -0.11823530267948787, 0.07230656575600565, 0.1309510824022141, -0.23243381550435907, -0.07336116527460344, -0.06784266276551598, -0.23767799896288813, -0.24649670289572478, -0.0607610235597086, -0.24100577759250816, -0.5340435086879899, -0.19801010911258618, -0.30530365563195994, -0.473789728968158, -0.5201007437277357, -0.20479618454306892, -0.43815414306071293, -0.24625214100079604, -0.5214103554060842, -0.5454735810439792, -0.394386571048785, -0.1350402108998929, -0.5073680247668514, -1.7534958239787035, -1.2138875527412034, -1.384085068837026, -1.5071726714036047, -1.316857256418706, -1.260000653857741, -1.4689204306945303],
[-0.07484458368919203, 5.897471424274778, -3.3289318575972526, 2.8957838300969803, 0.7977152479888322, -4.602314535988241, 6.803282677881078, 1.114380175851109, 4.168648062530696, -0.5423271961192712, -0.10122011978646127, -0.24596814436900172, -0.5978785660942684, 3.153350029224356, 1.006250877359115, 0.6017970335518723, -0.3368227473182097, 0.6206984056056126, 0.43913729846236754, -0.9645096447149992, 1.0039659165922055, -1.7864943946064762, -0.40012152326026523, -0.6935773132105607, -0.40097603967887474, -0.46547790454512367, -3.461645315362651, -0.49045428247449774, -0.49112233682844225, -1.5495113069848292, 2.1904681190108297, -1.2028167606756686, 1.3821544818250675, 3.1323558906262914, -1.67163594045486, -0.32865666478159283, -3.3339789796288652, -2.0012926976324734, 0.23756870536099964, -1.1365874213855016, -0.5598024417548745, -3.9429478822692534],
[0.08483114318177545, 0.18903505841929552, -0.40299375653814135, -0.48193039140825317, 0.21479460885826698, -0.18255053774862992, -0.13776945912882418, 0.09838603003788135, 0.10261854769204257, -0.560369595814125, -0.5022364756388685, -0.146974725249779, -0.10224463962970338, -0.3339553679564566, 0.06409278226356803, 0.14396210977474272, -0.3929934450297438, -0.4909262393484382, -0.04183462740043688, -0.29789537557593876, -0.4668374607755278, -0.17511520703194203, -0.0910914395603033, -0.4975699535720463, -0.3288625417703187, -0.3550423281921908, -0.3613108070022456, -0.4855289381415102, -0.56732183651199, -0.5752741319646175, -0.3139869621958956, -0.2756279288400774, -0.4935642132146542, -0.43996142041360337, -0.6672854183666831, -1.7295268335339913, -1.1056338503131942, -0.7510468271661537, -1.6282683094874106, -0.8649467768297836, -1.0318437246734478, -0.9870259677527884],
[0.9964876871440672, -0.27994496563432897, -0.2735103794970629, 1.1129255376811882, 3.793069831564496, 1.7604534463131254, -2.5221373382672923, 0.2521804179364076, -0.6928337654775523, -0.5854396623697068, 1.1474412885393386, 1.0772664817946813, -0.3978248666326584, -0.4717213835367939, -0.05664212373849926, -0.24797185686461826, -0.27319900857959173, 0.32058959834922307, -0.0659073181573084, 0.45779213679301767, -1.3834517218975289, -0.42205019818047884, -1.095808151633786, -0.3280347026256412, 0.40537912070412363, 1.2073576329968057, -1.9005043082947224, -0.5062546000849341, -1.3293719561938402, 0.02487276291640912, -0.39836018497554776, -0.8991606736318052, -0.885044440110976, -2.3305171597005168, -0.20027191426928945, -1.4281618421044548, -1.259689584066978, -2.594240251847382, 0.1448041657189999, -2.950849596109654, -3.4859158193530706, -1.2309707239504128],
[-0.1674709024396409, -0.2407096326721345, 0.2048222133217079, -0.17775019404948675, 0.18725442010214238, 0.18272088960733082, 0.10785078616857018, 0.022006428113963073, -0.29025584302979046, 0.0612873562884691, -0.3570864122442428, 0.09030447062011994, 0.09899756484866468, 0.06861500284466239, 0.10309854978432477, -0.22256386143926243, 0.19182587589735656, -0.12748437636796797, 0.18479680501472787, -0.247441975912213, 0.04453611773521361, -0.29957811583908406, -0.1888913069536201, -0.17766073973004587, -0.22880361097215354, -0.0760005970297433, -0.2535844751183295, -0.30233455483216665, -0.7027780421522768, -0.4737645775859888, -0.473675854183606, -0.30260150902473554, -0.5350570122047537, -0.5758926474732194, -0.8773294995003426, -2.8812213479012634, -2.5065216394944856, -2.7630374194209915, -3.0052841096198284, -2.9279498859718354, -2.595484142282725, -2.914028174847895],
[0.044866943730851386, -0.3170337292578387, 0.013797769245598505, -0.10140346757606129, 0.37407251514345646, 0.3401425001261267, -0.23116635430559823, -0.1055085140207429, -0.15470175355891697, -0.13595253278368832, -0.28832920722096284, 0.09792214810918258, -0.11096999578031327, -0.20966115246002467, -0.13490265906650173, -0.15526001321978097, -0.27344570020162806, -0.18460455053343958, 0.25110236107678247, -0.13055471917283623, -0.24375394496370476, -0.17342732187361218, -0.18904925125635286, -0.30845631919531485, -0.2892339859404512, -0.12693218842014492, -0.05374631471818311, -0.17836948799813143, -0.6623409246047436, -0.40489969610815235, -0.48021552375900783, -0.12772227010462403, -0.3751273008196629, -0.4104655481642579, -0.9422931313757805, -3.0521880205565637, -2.6731282062382853, -2.809152378551741, -3.1093156879809376, -3.1736658791755534, -2.8065338869977943, -3.018668017002595],
[-0.03472802901634093, 0.02917656401771934, -0.3015973769691348, -0.04030652376726945, 0.8684262449391226, -0.07709014208034674, 0.4024756146041799, -0.22798081637824152, -0.42098027286933243, -0.23688848952698185, 0.038531114747073825, 0.6679849025051622, -0.35397833488515695, -0.08495377176665972, -0.2218813272817162, -0.6692669484286294, -0.07910284589822185, -0.11561620856483022, 0.9983965192272191, 0.1388387277569125, -0.1039269306337345, -0.2345294385860682, -0.1830869077709699, -0.07851123380956804, -0.06405671688263968, 0.02680824769988289, -0.10863059663771242, -0.028658206627529853, -0.7097358243353722, -0.23629938431257402, -0.18481578566848625, -0.006424507951539287, -0.18224224017061, 0.344625062558929, -0.7269166906788185, -0.6817326943265095, 0.5506554890634207, -1.2162491793821548, -1.15732048462114, -0.09438900593848193, -0.978745991818758, -2.0940255327773474],
[-0.06554078411759413, 0.12690121882688069, -0.022649689581131872, 0.05956332583960549, 0.017812419213493618, -0.03988346256038213, -0.015015953439214427, -0.19460814918905456, -0.16043324748578253, -0.26973557519137337, -0.18716759854691487, -0.0009015450034742734, 0.0275617834124011, 0.01911972626339448, -0.047578106291744894, -0.230122662279872, -0.23669516831388313, -0.32438446553856104, -0.2796584143605587, -0.0817901322261166, -0.11882664267635325, -0.3291734343061764, -0.45291521887648867, -0.277046885557649, -0.2792812904911025, -0.17574405970750598, -0.5118892359269974, -0.3206110596664619, -0.6697995733851063, -0.43909781566164524, -0.622578496904367, -0.36053194417215834, -0.44560356082795716, -0.22643089781608952, -0.6776576263794151, -1.5729191419032231, -1.2448080063300866, -1.3533664456828567, -1.794856644685944, -1.2239041753603364, -1.5475042773189707, -1.676050709821811],
[-0.5300963837830234, -0.4323278091393722, -0.4276985735998445, -0.3590155078554396, -0.15485506761724252, -0.29675683054251023, -0.22435202894609943, -1.2183097937623522, -0.4669052563320209, -0.8631263922460461, -0.3247592317849791, -0.5455710799745453, 0.15189177820298558, -0.5717975275574142, -1.2695676246619105, -1.905966247008077, -0.48705891950729113, -1.325575837774467, -0.7895158666643524, -2.1456341603819196, -0.3982258912657394, 2.0775354272449826, 2.228683193123603, 2.85444371516643, -1.3976405997752135, 0.515628294517317, 2.925603305647719, 1.5495748479427718, -1.2299044286808727, -1.0770748057608774, 0.013967264312137876, -4.846924707792362, 1.3074957002827365, -0.9748272482235614, 0.2857551881383555, -2.7531848763769635, -3.1674525944079535, -1.6609250527687331, 1.2552447980477965, -1.0115231289793623, -4.338066428521645, -2.4927981293281523],
[-0.10270179368835725, 0.33603710358415634, -0.029743451171120934, 0.07442723338866307, -0.009680129512758763, -0.13993778211451272, -0.03186501740386729, -0.23932670803951722, 0.08159655269774407, 0.036433337638473645, -0.20063452330964923, -0.2898896320761882, -0.10275445819250886, -0.19357238233904647, -0.23772732356697443, -0.19638820675315824, -0.24394779258926932, 0.09551697837703951, -0.25316971221239487, -0.12410442054256911, -0.0117495699179998, -0.13566855261606917, -0.2769210410918309, -0.3079322490968512, -0.30228650889206315, -0.09672966152717612, -0.38555265526563803, -0.23042860237977525, -0.7597174584791053, -0.5349265815070602, -0.3118203575820214, -0.4575494336801607, -0.3376986620926405, -0.3913074029836707, -0.762164083762189, -2.505712822613976, -2.2114232520606287, -2.715953412645856, -2.544820633766681, -2.527958890711248, -2.32457672366989, -2.4755463570121994],
[-1.0286325521655149, 0.35186093968324406, 0.4750614849489824, 1.1470098145170557, 1.1103883520067164, 0.30931133495978885, -0.36150136781250436, -0.5351091821621471, 0.11801975802693335, -0.06738718629463611, 0.44795594149509693, 0.570276888293167, -0.0883202194908132, -0.0176970467533818, -0.4149766171915706, -0.04253288223589385, -0.4852250937949433, 0.23019211790394553, 0.2275146515207629, -0.33345963722639593, -0.2225467449333465, 0.28957981090764856, 0.1868994101373807, -0.39140328939840213, -0.3704495141400885, -0.4742024998499371, -0.5695981920574968, -0.01084978484172771, -0.3654450895071777, -0.4177239824639062, -0.6409341249119607, -0.5702450443325751, -0.6947334942738556, -0.5493732356179405, -0.8278151606500047, -2.789595412398874, -2.610021059509496, -3.3410111567653673, -2.626006045170235, -3.2916953479132367, -2.7809507339142994, -3.229159977573498],
[-0.006688827011555087, 0.46347935955384273, -0.08980223478752827, 0.053482706397189876, 0.2549096806964834, -0.24046387205450692, 0.46274224490418087, -0.008612326693527766, -0.18853153344657975, -0.05872064999606434, 0.0793360682094574, -0.05155045802586661, -0.3093967726876023, 0.15003239688487474, -0.24728729591781626, -0.19012142385038244, -0.3516585709884277, 0.05491044373210839, -0.10096609595280917, -0.33588036261243664, 0.08614615511224401, -0.3240683410343306, -0.3411073129212387, -0.5910923238035715, -0.4770152659599383, -0.038082512231918705, -0.5821215735367715, -0.45269585899550085, -0.8474797614594702, -0.32440285249480644, -0.548833544460171, -0.29017644103552526, -0.5387259927392152, -0.3252247393048143, -0.7258653370120556, -2.309828042555265, -2.1084764855722478, -2.427384248071924, -2.608935936931154, -2.3427085197666155, -2.4007924329886134, -2.708742698170055],
[0.002711943587437131, 0.2545077533392007, -0.2460729832416088, 0.00964346901087342, 0.1536216389928008, -0.012849906558753278, -0.18582787122871522, -0.1390248743569531, -0.1320533956790603, -0.13110124614988414, -0.15587498361480892, -0.160650321472226, -0.0010474361744083399, -0.262967291230811, -0.10887504929305179, -0.26567771026692005, -0.2914092948707461, -0.19445484543596736, 0.0118868074000999, -0.21444467803681494, 0.23899521751161595, -0.23256194819290385, -0.1451070263266417, 0.09385533596252739, 0.348575166137442, -0.10149471571740198, -0.20018249279760955, -0.26390452575022744, -0.27250120749742773, -0.39182104512884514, 0.45899639535895653, -0.400701185086663, 0.0934997010634848, 0.396049183642046, -0.8790369871124295, -1.0765320912595657, -0.9597232690871371, 0.9945781827329442, -0.28384068607315366, -0.5758290540874699, -0.1288373810734049, -0.3451700472501719],
[-0.12778048678922752, -0.1658809560184235, 0.04289947169929763, -0.1081788641390652, -0.02219694660458527, 0.21458405861202515, -0.09110208054464565, -0.09355624209137389, -0.41910208820060835, -0.15915297672007228, 0.009722209633496782, 0.01437313072973064, 0.09793772033573835, 0.0937090227361486, -0.1051888019299611, -0.059757094619265566, -0.04789111833612214, -0.5755069432723351, -0.6257652334304746, -0.08973881344703838, -0.22454936337567652, -0.017328436793144133, -0.27855147436996003, -0.29411095071638277, -0.5493691533202141, -0.8283730926516122, -0.34989901732199213, 0.061346523670484876, -0.8047679047034108, -0.6452542760520318, -0.6596492216559153, -0.10645225932319523, -0.6725411813976265, -0.8743397551589508, -0.8157069064596814, -2.703221790926943, -2.4877728024230046, -2.611671406348435, -2.987999559634411, -2.7506720033932606, -2.4732571821998097, -2.585312819048916],
[0.08164600868625345, 0.13048098069847686, -0.14831948016582155, -0.1844670861096677, 0.0020771503783107038, -0.13010827758025678, -0.11851942521134837, -0.30002496816328633, 0.06133506571256546, -0.37898746669869204, -0.3742995211266473, -0.14176484302719994, 0.08992058941275576, -0.1540795507266429, -0.3839214001635471, -0.7028035673710367, -0.07869642377391538, -0.798205878557403, -0.21510495390401344, -0.33309778204307805, -0.5256533484482371, 0.06452824702044926, 0.007128123914966183, -0.302420544679477, -0.45062266092596187, -0.3958342939553505, -0.33708675030970503, 0.2755529434686337, -0.175856721953842, -1.1884337816787174, -0.4372169574431072, 0.15447159508492567, -0.4022944235916309, -0.931242438657317, -0.15173441138893404, -2.141034178972963, -0.5694467231524122, -1.6185983636902326, -1.7840312332350587, -1.359740736309138, -1.363394283709041, -1.3311259244629008],
[-0.033537210836797726, -0.1264132016057463, 0.17903613047915554, -0.3756888722714399, 0.3877282697500678, -0.2611332848827832, -0.10957418367362254, -0.12199427076119766, -0.07411673402207748, -0.04090946788202098, -0.5263294456875867, 0.08763167625819623, -0.2103509301153987, 0.020253019782472827, -0.14450692444685737, -0.4523189703873688, 0.02241878141643129, -0.511828171201946, 0.045228848830643535, -0.5092969715452723, -0.06337686517278074, -0.6403660795771947, -0.592802051111822, 0.15635582951760982, -0.367118363691724, 0.41571028130542986, -0.5037285022481928, -0.2874741926011249, -0.3071736104697141, -0.728585339402237, -0.23999089082172967, -0.20680585458357142, -0.03780768679993316, -0.628076049692734, -0.40563095264736365, -0.8328820432637535, -0.4085974002058914, 0.48537606453539384, -2.1823019467472156, 0.10794011258529654, -0.2073117115416487, -1.1038189844581106],
[0.23431842176082568, -0.2571807603557187, -0.2196286120200673, 0.08171091206371828, 0.1673724291729648, 0.04116394172936554, 0.12300450977923723, 0.12902725803471127, -0.26134729145060215, -0.30007835750879347, -0.07384844254329967, -0.0801105562273881, 0.016019139757032334, -0.014837575650793784, -0.25973455386089406, -0.22296466704529083, -0.06537451678202613, -0.08203033412272141, 0.17177199702896218, -0.327262519873311, -0.12122112893038169, 0.05239994263482772, -0.4196182980638037, -0.3480015895522359, -0.2637178962825707, -0.18819525279291963, -0.46782405606808547, -0.44361333893904387, -0.4481869664033577, -0.4340084093041325, -0.18192455154505918, -0.2600652786694891, -0.47249293394714276, -0.6361547073955895, -0.7177740369367506, -1.3658014214377039, -1.4602851908175078, -1.3305013141381417, -1.5050207034049208, -1.5890479549227323, -1.2759353607999397, -1.8670120975511506],
[-0.02078806050573598, -0.39797794545598864, -0.8974041561557102, 0.44493255183569946, -0.02303016188532469, 1.04771078511687, 0.638918022863003, 0.20443439309456768, -0.6382034479047209, -0.889388300047784, 0.11106708510053555, 0.04003738740954899, 0.22325387277679684, 0.7224996515688862, -0.19907609981414373, 0.056013120664277556, -0.11283296871173905, 0.280937641626878, -0.32320330821671833, -0.019429517344093962, 0.05160178673008161, -1.0968979696856287, -0.9958149309679897, -0.26189452705893557, -0.5779343443511133, -0.6485501436729886, -0.47440525713802384, -0.26362526517749324, 0.5608881727523822, -0.4447697399500101, 1.3985539945209808, -0.339746955714622, -0.39055524758585347, -0.9361196533157399, -1.040545412639002, -0.1320154798729344, -1.105153025995388, -1.786993415548102, 0.8537827053859451, -2.5054293550978475, -2.2529074248556227, -2.6506749428482683],
[0.16822820198310628, -0.11721843847475316, 0.046797150550867556, -0.13415756518311878, 0.23079434294118045, 0.10741067591266316, -0.07471605693862518, 0.06234048513488375, -0.1869450300027459, 0.08892361569840732, -0.20873826654982483, 0.2357958824926688, -0.12813399301258788, 0.2510730544241695, 0.37418981041678673, 0.5050775739041745, 0.15480913574651872, -0.09440444624283575, 0.34700704235962077, -0.2088014654578045, 0.004822475418930515, -0.015544448089373615, 0.19852052863481312, 0.5931934370688433, 0.04608117977733943, -0.294536667338356, -0.5300885097498552, -0.29536919711698595, 0.9965553152846152, 0.06872904083647756, 0.4878192027087766, -0.04986951845788665, 0.25710392692058137, -0.8471265129913184, -0.5266089601259261, 0.5447348559464453, 0.32725394042617445, 0.393932021049673, -0.7270086942119381, -0.9309521811169501, -1.3646932396413336, -1.041662249027099],
[0.20183365651656693, -0.03249706847185701, -0.041355953661045275, -0.6252276985646801, -0.74085316687481, 0.41043237501705127, -0.11336062105910433, -0.13788796527260225, 0.01767356523226802, 0.18500408803701074, -0.3157552738998561, -0.4774872940958943, 0.7516554287258652, 0.133000030465825, -0.31360267284355114, 0.02741166518062024, -0.2549904252591943, -0.45137683429358477, -0.5456207563979895, -0.35251274011171446, -2.0702470699197497, -0.6309853121331535, -0.36334121535144154, -1.0841018258632662, 0.6424359315404302, -0.9764484634350654, 0.16283152025138145, 1.7961838402191008, -0.9276234965235899, -0.6163123306672401, -1.599281550711555, 0.3487765079326009, -1.8542730895826873, -0.5581076485938119, 1.0359409720349106, -1.3910407514870524, -1.3005578928873176, -1.4559289430403475, -2.4610322572862935, -2.4727209791256146, -1.8408600831253805, 1.5400272931839014],
[0.36692621402824294, -0.30002258141722554, 0.3045543195258802, 0.3182027596533277, -0.026369356759751793, -0.19352366524673012, 0.22769577676744948, 0.09189469003000737, -0.22431318890628968, 0.2894911309705567, 0.0401302472349237, 0.014768491173352992, -0.255544848057901, -0.1303768726763952, 0.046871524831352734, -0.2704764826288213, 0.040711966588628176, -0.15376428158289615, -0.16184278407582262, -0.26344367181156514, 0.1544587346514287, -0.28088639370065444, -0.4579966510438463, -0.4476313559415836, -0.47171386301327195, 0.25121082606325934, -0.729591395614343, -0.2246022882693272, -0.8529375121947828, -0.6651645618292629, -0.7852716814063208, -0.3067759311131516, 0.006996416496152811, -0.4790751939382114, -0.15866280901361504, -2.139983829971149, -1.3252828325489436, -1.6724357881053704, -2.2228191796019168, -0.9223037357529342, -1.1680034881875416, -1.113971413799865],
[-0.15044283208159992, -0.24129100098778122, -0.517938329780244, 0.021205006308021686, 0.563683691604265, -0.05739451811390354, 0.8314381803430106, -0.11182307924261733, -0.1876813248925165, -0.30934838628149797, 0.10308229602896476, 0.4319048355430384, -0.08470825652443514, 0.5201334230715202, -0.4053702238588136, -0.01979705241765834, 0.22936508772137087, 0.2578795765922686, 0.3374240980940696, -0.06435800321467497, 0.30424976471668647, -0.2222463984310755, -0.5471803913842906, 0.008482774957747642, -0.4873900605504722, -0.03152470604553802, -0.3139587784690813, -0.0975171420373966, -0.160389269174939, -0.18706367624288656, 0.24768914036847656, -0.13861258403017976, -0.46767026274999407, -0.6224881161561977, -0.6099176053097831, 0.2577739136260904, -0.4107354466522464, -0.6130370623889386, -0.7893406563668846, -0.5898446271400186, -1.2665933336165602, -2.1732105792781478],
[0.3760225442627033, -0.2140861661723924, -0.35918133593211615, -0.5056722805787243, -0.37081303118039244, -0.00927317229588215, 0.032535255399412814, -0.052410180371447616, -0.23522273488987006, -0.20011244498992012, -0.6273963274888534, -0.2292212481789846, -0.03915405315873407, -0.2701007622378319, -0.18396374445710212, -0.6108912475086367, -0.2754280697329018, -0.8113366126401028, -0.16282003205858991, -0.5197212698089556, -0.19933704050575782, -0.5585813183790306, -0.5548555074615856, 0.3946220574416875, -0.4231044697245628, -0.182683198248362, -0.3872271266264958, -0.26828160437201776, -0.7666756596003762, -0.9256706068063822, -0.30938153673886215, -0.06679215848502003, -0.3706521623518438, -0.72730040896041, -0.7362208576150111, -1.8206245221450126, -0.713712447288782, -0.7555708696582986, -1.855585302561881, -1.480224967981747, -0.07558058676689931, -1.658037062406927],
[0.5417032091169013, -0.2597441321996567, 0.2375885443018634, -0.5519805614085151, -0.3111050190809463, 0.23271354003999947, 0.4184907367406822, 0.38987382638743173, -0.027154747875184647, 0.15652066793062241, -0.22087276598487834, 0.07536636215508913, 0.49694387210301655, 0.17558205342757346, 0.5245798519938196, 0.0668955716515241, -0.1633695597448986, -0.687074697372038, -0.4993237420103969, -0.26306432098484406, -0.21110457444656464, -0.1068747126105244, 0.05490107685264842, -0.6206427916535249, -0.28878948630649487, -0.40855961017515946, -0.09688617399802898, 0.5128711181422742, -0.4016762057191623, -0.22961228185401614, -0.8624376420112925, 0.4914001446237596, -0.509552654140457, -0.6093030523810676, 0.36023650028974097, -1.4536540482081952, -0.8027710943847168, -1.3138301517962965, -1.5101397926735283, -0.7953429170173822, -0.8229448841985315, -0.6316171953451087],
[1.672423228234801, -4.58174627283788, 3.238686533455195, -2.0283278942865604, -4.772116982829872, 1.5811799666344357, 3.4914633123529097, -0.8241313231612568, -0.03309613722856903, 0.4773768153205158, -1.5812542021493432, -1.1613114791197174, 1.8159324653274542, 3.5392156035770106, -0.37422094067058836, -1.5341570249736376, 1.324204733343717, 0.05235558956357372, -2.0095732108524724, 4.315661530801939, 2.126104532135577, -0.0761690338525492, 0.5895990590245588, -2.941227794328847, 1.8749912740649615, -0.9307189341373707, 2.0839394122024393, -0.7695063640002258, -0.6979246273434001, 3.061542124759974, -1.6808588797849848, 0.8340936435427596, 1.845257720847155, -1.6877071323662338, -1.8265321942792516, -1.0987915340603227, 0.8433456079745152, -1.8469109483449713, 0.22179845503188078, 1.3428108439755393, -0.5577584497158881, -1.008559924474312],
[-0.8777656019580486, 0.45366442658318634, 0.22901750085090758, 1.1605986765075935, 0.7910300344139368, -1.504140010474775, 1.2131835827170314, -0.35765425965661657, -0.010366045047174881, -0.0860360954285244, 0.36331659375980324, 0.15959587743824294, -0.7536338523659, 0.9802216283415438, -0.27164290498701843, -0.15715021514519947, -0.2822779404114972, 0.4785815401396977, 0.13189422081336857, -1.2054737617055442, 0.6101326981181718, -0.11658611225232296, -0.467797014492479, 0.254822214029558, 0.4808951133526388, -0.42734483863794664, -1.3604589472973228, -0.6687783514417407, -0.567126415998904, -0.6830252823399066, -0.5397897505105712, -0.7830293634035922, -0.6023292652806375, -0.5098634906203147, -1.326182984258002, -2.8416766837556504, -2.6850653891981398, -2.699468547550348, -3.0691583670349423, -2.7867985181418544, -2.7978779504272127, -3.043733374213797],
[-0.11994865308544082, 0.10205757206856123, -0.03143021381483657, -0.18191083446308365, -0.0019466089813677, -0.09752014355121816, 0.154667656969711, -0.0569396425677088, 0.12694011575545877, -0.07352506840743049, -0.3404991911885858, -0.36480601769372495, -0.04651285744694027, 0.07864265726469595, -0.05547110187720687, -0.07331116687339585, -0.048092425827383975, -0.2722595104102711, -0.42866581662769776, -0.11831063662344865, -0.04999784882916509, -0.0416349875844179, -0.250278170328877, -0.22210893165407888, -0.6243478762759123, -0.38012604107493786, -0.33759836662857606, -0.12461930895285162, -0.5582162280873204, -0.531173906696585, -0.1949691792652764, 0.03574501696565357, -0.5499271570453769, -0.19814496990951488, -0.39174007232691843, -0.8043200567341805, -1.0309626750580387, -1.0178302474686092, -1.5825198950310608, -0.7132445753834004, -0.7620092518924121, -1.157542451721261],
[-0.18858500030911332, 0.09423335336324933, -0.12657332712625924, 0.11285517480567105, -0.059551003289330814, -0.02274795151571172, 0.004520055292872384, -0.06752727862841729, -0.12476866402806362, -0.36805727688409623, -0.17645578740393494, 0.0042195721069851104, -0.23300406395040374, -0.15956653244197466, -0.20327734562407926, -0.04597017214983747, -0.4377342999138832, -0.13221977314877634, -0.014111303843120707, -0.4195821188830829, -0.22207539384555208, -0.3326805683789308, -0.1616718637729109, -0.2694862265016851, -0.05519232616097038, -0.2207484110192644, -0.68117592674436, -0.11022985588647026, -0.6325901566510189, -0.41278224474253683, -0.48197763567756763, -0.44645333621405653, -0.654881201206947, -0.6841195303912184, -0.7572029088700913, -1.5055560330293873, -1.2325941971397212, -1.8348859653152014, -1.8756537650499692, -1.9230223402733868, -1.7532181328802747, -1.8956984386283844],
[-0.08361773059306252, -0.32980927005444116, -0.10036362158576985, -0.029706342959614693, -0.022088650676400586, 0.17035599122467257, 0.23669985110888878, 0.05031283449829135, -0.17869573386908905, -0.11911445139350658, 0.19320272672801292, 0.10033131559464546, -0.010757870022113591, 0.09443825295625852, -0.2113252164573425, -0.2092718087068344, -0.20807586633571404, -0.12438297325374083, -0.065271373590939, -0.16691628583233326, -0.15102483740518083, -0.013711727808671139, -0.25341624889676184, -0.24706338036584685, -0.1373134969847958, -0.3752641154220549, -0.5129412111909389, -0.23973279719556292, -0.38071246502647815, -0.5551365908865792, 0.019785575550536283, -0.19650876714560728, -0.4373997253501833, -0.8725008841335937, -0.6903466817359025, -1.115648469909864, -1.3703066089986533, -1.3328265717681966, -1.3778317630365422, -1.580199718137268, -1.3509423602455384, -1.63772747026157],
[-0.15144356849478277, 0.7809456789034296, -0.04259805734471148, 0.7906560545236321, -1.2643111928509727, 0.721930753939881, 0.5245775447369516, -0.03852416066065632, 0.3628840911821149, -0.13060746903659376, 0.22271960721408898, -0.7396461466958427, 0.145345184175854, 0.1909428340913963, -0.08909878357875058, 0.14719461133879103, -0.2713231626101298, 0.6471497358632767, -0.7895497468353482, 0.03618718436535301, -0.3596592325145462, -0.1397766592121906, -0.19293076037639678, -0.5722305894732541, 0.0856484576292622, -1.0891679833525607, -0.7048137120355324, -0.32025386393429245, -0.8637027684705039, -1.0209422438058833, -0.13045334358913824, -0.14694644873057106, -0.48932400739792403, -1.2953441496351268, -0.870849070505371, -3.3852989691489954, -3.049288417494888, -3.445255155906083, -2.567577864374295, -3.6614508561495533, -3.2569622728164362, -3.602038956418579],
[0.6132548133198842, -0.46605826061235384, -0.7095817812407621, -0.09345076950927289, 0.9301227771867604, 0.24064828993066792, -0.10448815726694384, 0.1461988698265634, -0.4522728533324736, -0.6424415939341026, -0.28922668857042355, 0.6480102071043224, 0.2544760897695877, -0.05749432003916663, -0.012948690256291117, -0.7242111114961319, -0.47276886234471294, -0.41201499782363227, 0.2597552729371028, -0.19228483445412345, 0.18557356591274524, -0.26493486425768975, -0.4687058953547723, -0.8213923877071135, -0.12187039933001477, 0.047277702094865065, -0.02667001184358425, 0.052544226012244184, -0.46556196253450194, -0.6738979893475169, -0.38302113998547027, -0.3558972752821949, -0.7395187622256565, -0.4370318908342905, -0.7085100712544969, -2.9828482653192014, -2.2995193684756807, -2.6010923567796254, -2.343699885313928, -2.6329524591763502, -2.569215080426405, -2.8672827522284297],
[3.172940600470381, 6.819170607538103, -5.461642029567879, -1.9724499148462387, 3.5619746122386946, 5.6653308673554506, 0.9877178769503424, -0.18903382496758372, 2.145807415505388, -4.019627554271012, -3.065361176776966, 2.3206352138594095, 1.7982719644848524, 1.4206212371566873, 0.5095028787445887, 0.7279123665554607, -1.0599415499190379, -0.8679902155812507, -0.4485810951904029, -0.8900612015590276, 0.5564817647373452, 1.4339203199140667, -0.04712559247008706, 0.22976969799978128, -3.559463104038347, 0.16515527940099425, -1.7864982190140204, -0.8761317067174039, -0.7343187128166365, -1.9802235562336346, 4.749604571446465, -0.37124698376348975, -0.6409815654670113, -1.8025972402295167, -1.8174038460707975, 0.36550886736899385, -4.851018990596929, -1.929682138186446, 0.3105233898963407, -0.852665573748077, -3.30362216776292, -1.503290201564665],
[-0.5030751115978668, 0.18542592651225098, 0.08945099028795916, -0.060234999873865006, 0.13306898011397972, -0.04860527964207302, 0.1834955271641552, -0.38472130328550413, -0.2621301014333918, -0.4516775409890386, 0.2524082729783874, -0.16541280339515205, -0.2447582262648939, 0.07353951683835373, 0.19975690031807253, -0.5192349566799308, -0.43268740357378155, 0.060038929873807836, -0.2636520227210782, -0.32619640389846044, 0.04757411076235131, -0.611660212431473, 1.090402520706962, 0.1002827555518154, -0.10767663304662074, -0.5989495426097857, -0.3544968954896457, -0.2669980820406147, -0.8696594385354335, 1.1798054387421035, -0.6545715323650046, -1.3765024735450406, -0.5661096177196909, 0.05133888371023063, -0.41444673333641324, -1.9476378268583918, -0.9828572385053403, -2.458487886040321, -2.7494681509738803, -2.105424582748793, -1.6218288259105775, -1.7641375583493342],
[0.08303100636959383, -0.059420349293648686, -0.07572251038779121, -0.34384025075538865, -0.17114772026284114, -0.10192407918054185, -0.07136075783313811, -0.20813668650841902, -0.1302388650754462, 0.054014495131277984, -0.1837167539803095, -0.1618471727248506, -0.05409880977263019, -0.2362266837238583, -0.03289141424154408, -0.3068336361243669, 0.1569448617614308, -0.3424369343034005, -0.05138907720201926, -0.19670677312810123, -0.13308341529736079, -0.19548744183920266, -0.1491638075169601, 0.07627525322734298, -0.5812252819865824, -0.3964959655784075, -0.3084040916929852, -0.14684198101030077, -0.7798082605227086, -0.6110152198878018, -0.6018874294789659, 0.005616914836750583, -0.5033779900951679, -0.731683053375419, -0.5820192781382604, -1.2565276180631504, -1.278513254931936, -1.1134008734601033, -1.9867525179953605, -1.1619871424824444, -1.131465514068132, -1.0568428094054438],
[-0.01423778508274946, -0.15021093321927606, -0.003819121739683075, -0.25959449304467397, 0.06528389181324501, -0.03887606132643041, 0.08595754533403577, 0.08445635839784299, -0.14066389591260886, -0.3303681946418761, -0.12697416808316817, 0.16146251008396303, -0.0955077740650872, -0.16896620697300424, -0.6855059275103345, -0.061604555798767044, -0.32132032571101016, -0.5506875131659721, 0.23263295698842903, -0.4907884279209403, -0.22307527858840157, 0.41548369457922296, -0.5489960294626166, 0.04650565067203747, -0.1713089814441575, -0.25431937019833056, -0.25770771564596584, -0.1500126741110135, 0.6101158668163311, -0.276531940590969, -0.6793615065314331, 0.23583109918993278, -0.7674017882810303, -0.6733098996499127, -0.6244898237520923, 0.13174468671119682, -0.9931567038806354, -0.9803713031342449, -1.424074075495531, -0.639116417975549, 0.3009673269022826, -1.186843640189667],
[-0.5352764137836694, 0.49617485983316856, -0.12836712712029757, 0.4765403227860597, 0.15899311641234162, 0.061676730499593026, 0.12558519282002795, -0.09118115710139035, 0.045857020963361306, -0.27771686589701194, 0.29659372836845266, 0.09313662137637296, -0.1577037036077009, 0.07380030686300199, -0.20828175564301868, -0.009139263590248237, -0.20347339674107767, 0.3379144810711794, -0.007986409004314507, -0.1600562689105575, -0.0057999609463735904, -0.5000834909526006, 0.43081830809279864, 0.36763928253502315, -0.07766649119249609, -0.2349063392415532, -0.3346200081459793, -0.2728726104247818, -0.9560552995818513, 0.6317574680116929, -0.31251681056062297, -0.876037255842098, -0.015629762007816815, 0.309463807061799, -0.6819972700332234, -0.590054673473005, -0.20778505729052385, -1.4977094526216814, -1.4480415075408093, -1.4065163885489818, -1.0393216215117203, -1.6134473686128015],
[0.6436375998749965, -2.6931371457920643, 0.7290151829581776, 2.485593356586888, 5.813238069025601, 0.31228427165797323, -1.914394090296786, 0.5262308029912043, -1.4238967574172148, -0.5164518849022237, 0.38385003172104115, 1.3938774523833486, 0.5901282413776815, -0.6528823629097207, 0.2519800107245314, -0.9734753787187526, -0.2991106827187457, 1.2591241475046218, 0.9015796071556317, 0.8161684310690428, -1.7029742867108137, 0.6874468072720729, -0.7491361997110737, 0.06620829831404512, 0.2777334504133553, -0.2950744093876644, -0.128244442550649, -0.1954831487654459, -0.1867107928703499, 0.7096977653969926, -0.5452584049346024, -1.619783679066275, -1.4472985809133625, 0.8764580469503537, -0.3599968876186491, -3.5426417739235685, -3.0796238493565418, -3.1136132353807264, -3.3874735275562458, -4.064940697634112, -3.4586875955199403, -3.6694044663730523],
[-0.062213118691529734, -0.29397844784274263, -0.24434593167251545, -0.4345176252155828, -0.1705333939167274, 0.03827976148092629, 0.260248802044176, -0.1795813605657401, -0.23013722397191647, -0.10316777075595236, -0.23530941105329967, -0.09185876800634525, -0.14686633369739627, 0.3233741465536884, 0.0031470000147771626, -0.2571027634817779, 0.011619524894151658, -0.554811629845365, 0.08076132926972107, -0.29360754110511855, 0.06749146109840627, 0.013196258061560478, -0.14269997971698334, 0.022488303744452674, -0.30349612741817167, -0.3213295591701377, -0.4553234644425422, -0.21521074277905797, 0.0723470112431264, -0.665513332259114, -0.20641638721137198, 0.07437493687012635, -0.37321793631404604, -0.8023949218293083, -0.4101404170561907, -0.8604135777640634, 0.003031420906005261, -0.7224225033113104, -1.5266926716510778, -0.7458805574351406, -0.2193262040466436, -1.1957413044297067],
[-0.09899223304456935, 0.1664818413822352, -0.2718076835915095, 0.07637976740311853, 0.027722654150039244, 0.09914984867581993, 0.18762969583002076, -0.20921332428865677, 0.1332862471065486, -0.22410209900897135, 0.03154146522104683, 0.12065146108578188, -0.20322347349863945, 0.13287132517967185, 0.053482131015599364, -0.03693581319432765, -0.1283110698813646, 0.0880405219077509, -0.018561377736124495, -0.09796114771111382, -0.18805264347970982, -0.23840649821218585, -0.0903699515124264, -0.23542587487973324, -0.4215055274142722, -0.09086056879148197, -0.2896384745203554, -0.1491562693521976, -0.8197776402556677, -0.3422604728450906, -0.29566514786437237, -0.4761643839966759, -0.33350284116803514, -0.031683874921689975, -0.3965034073055457, -0.09661155995222126, 0.14036687209956805, -0.5229447706445, -0.797122887771025, -0.6606454443396421, -0.6920112952382103, -1.163439534892664],
[-0.14220847266542, -0.5327352148992514, -0.3746698758845963, -0.1412836966283123, 0.11123108543373032, -0.09622996606912361, -0.043181014509669664, 0.6401091441170675, -0.15066508804437964, -0.8429297982438404, -0.05320555661771826, -0.1902319193272829, -0.5994182803593656, -0.20207729969058402, -1.7668933036511654, -1.4157469470532296, -1.76958207181973, -1.1231448974950287, -1.7485993599337384, -0.4283991839022897, -1.4240018202804883, 1.8275385911114548, -2.2798826385155344, -2.0855384302177056, 2.5276235802561526, -1.8744009087548434, -1.7691896360918962, 0.7183927170464229, 0.9120311509924571, -1.4804762346069473, -1.3346697107137897, -0.07011428618148788, 0.06523740330925618, -0.9405383041577488, -1.1073416205193447, -2.9536561025413985, 1.2998044120906533, -0.8662959637056316, -1.541303260755565, -2.072874902305211, -1.0224851008411657, -2.1291761977076757],
[-0.05582669930163672, 0.04884750054278984, 0.04832247758463263, -0.11407569073915715, -0.20166519772697628, -0.1511500354241953, -0.1504782828408545, -0.08082739112324883, -0.14835248397157794, -0.07174108567534311, -0.17164602671721343, -0.2272136021772246, 0.11262793316967584, 0.04644227358801109, -0.2741952654687009, -0.2902879931191531, -0.3036298304848512, -0.389322661915893, -0.20445100865394628, -0.30893410299695195, -0.21675110758454313, 0.08374013889446169, -0.044624332394954606, -0.3109101613008298, -0.2523377781038115, -0.14567231211135565, -0.0820259958925684, 0.02027359254167094, -0.3762090975193754, -0.5763214763279242, -0.5539782126764811, -0.08422054511456196, -0.5257178561390878, -0.4059016064227266, -0.4191378187683083, -1.144602450606435, -0.7721701486999671, -0.9924795372147355, -1.6538427760470404, -1.0140818230656756, -0.6656931700167896, -0.8646889926553377],
[-0.04104364188770339, 0.005207383511415489, -0.07704001140304542, -0.2221586377698567, 0.031388694901804036, -0.00873013679038014, -0.1869665151280072, -0.07544794240658369, -0.1758295791518532, 0.36803533326678783, -0.443591958719768, -0.17114275814389324, -0.029022056934548485, 0.007286084555039131, 0.154569459843548, -0.3491923492167571, 0.5083905034891406, -0.28628583791385487, 0.01241715883379128, -0.05038450366664249, 0.2888990117448146, -0.07550643895778286, -0.4749887444913702, 0.8188574327011457, -0.47533429288664797, 0.5173486672991201, 0.3837856891022656, -0.22155844424480103, -0.1511984585199952, -0.4430650655752872, 0.8476856126817317, 0.382551044496405, 0.5042130672152925, -0.10420670495421215, 0.02692525777342234, -0.41998410701044675, -0.16346231664555597, 1.141341705573684, -1.3642574799246974, 0.3440349838548573, 0.3208149034935066, -0.02012595242049694],
[0.012534966486624381, -0.29185116037676445, -0.06469585414041097, -0.437874258705397, -0.08049278337331117, 0.12205470544586267, -0.0629070242916762, -0.1691115431209493, -0.11496109163801232, -0.2786034322735366, -0.4168340427761059, 0.03357244906834237, 0.03257862246734285, 0.06445847652803381, -0.2547546450736193, -0.5405902846186468, 0.10953072546294258, -0.6410565590639115, -0.14952698569172326, -0.5438091265334442, -0.004413340155850561, -0.3919189712329053, 0.015725023064843868, -0.09215423178986037, -0.637282226617401, 0.0072220625455032906, 0.006860487658875231, 0.027952731697073838, -0.7264379040371893, -0.5520948719972831, -0.6345585871186663, -0.064823703325353, -0.4762966767528419, -0.8131165957584339, -0.6423064921416528, -1.833358486139498, -1.7636724907368373, -0.6725742559341191, -2.9951228287902625, -1.6233845704520444, -1.5472768668293362, -2.103565378005134],
[1.2773098942797334, 0.42352581487656227, 0.44543022759967793, 1.8785408959498555, -1.2146330654886495, 0.5046776543871516, -0.3406120410161817, 0.8092947380113179, -0.2818802720930852, -0.08687320802697869, 0.5296440307814483, -0.7783618104653486, -0.010511466946627175, -0.49744519645745394, -0.1687890098708056, 0.4416014602472116, -0.703138985640836, -0.36679340908765584, -1.0883713337670056, -0.6934164455672024, -0.5845078254997228, -0.37038164597481266, -0.32982919438144526, -0.5909132093155409, -0.3227589875402057, -1.2583727301077772, -1.00447399834767, -0.46482520117034903, -0.817379071209554, -0.10595647784720016, -1.059902039410898, -0.07822547175932272, 0.4811785476916627, -1.0504968834659367, -0.5948237037462819, -2.524349795559554, -2.2471353627753725, -2.3606262924619217, -2.1824812115328167, -2.135069598210735, -2.233873577480832, -1.3243584700951636],
[2.01138123962993, 2.8552283958471256, 1.9913098780720195, 1.1693984872377619, -1.514786007989106, -1.6488756926096526, 5.014264224058746, 1.0921704433423103, -0.18298025132548845, 1.125591831918641, 0.4522170206098044, -1.6603850758918388, -1.0592986275600635, 2.011676355156953, 0.8677801304691841, 0.6663354490760562, -0.5054374984668111, -1.1424772018265308, -2.410439854577623, -2.127691559078218, 0.28534042408561267, 0.389211245494714, -1.3935368400660866, -0.11237834296706907, 1.428705322855059, -4.4453898558914355, -1.758143832944342, -0.8214440531636454, -0.5087123881256106, -0.3874580483298871, -1.1136253819112387, 0.6507187332364511, 1.2799203761207996, -1.189914250243456, -0.19510770784592868, -2.5088886648750837, -2.860105965993072, -2.33548062244189, -1.7085391419990337, -1.1711160490962524, -0.15005484226203336, -0.9098450555979191],
[-2.036929977415791, 1.5865191117527047, 3.6931762940986244, -2.9847473438741288, 1.9198871700616937, 1.1510223692100237, 1.3387821647248717, -1.093682888746202, 1.59591701598088, -0.1739370093086979, -0.9990548604345146, 0.7163025156526407, 0.6823428848757994, -0.09134965929423247, -1.5534656600586454, 2.7584117363034872, -0.8985482000025334, 0.03253336331976134, 0.9239884327774348, -0.42268134087477194, 0.75840408349841, 0.19049086713379818, 0.008866095368846222, -1.7681661202192704, 2.0184878730925937, -0.9124046376512811, -1.4175110894791363, -0.4577703925940638, -0.2457784329292294, 0.04222940234090181, -2.2540816214962023, 2.3133025597909485, -1.1886017982387571, -0.30446271515172546, -1.4524122092963758, -1.9297000520546215, 0.8541640057858665, -4.214628921254348, -5.7358635699871865, -2.8042288108084557, -2.4111144044786594, -0.9563826199341164],
[-0.35147287364045043, 0.039066787452204564, 0.09200378550840191, -0.3549530939685933, 0.004947193257713842, -0.023013690523710488, -0.01919275494966879, -0.31883303511411354, 0.17213246421568384, -0.02590565970526741, -0.3881207574839731, 0.09115613497583572, -0.05921690630817867, 0.07797589533915152, -0.17218001290119084, -0.18595574230376086, -0.033133616378999226, -0.2880715917113593, -0.16856003191583815, -0.12403403036712705, -0.0875703262536489, -0.23701970113927057, -0.06364468299667381, -0.1597260574210778, -0.4153574930677556, -0.013009361810410918, -0.10343875622624399, -0.37297813897263893, -0.6914885743339111, -0.4933916514937875, -0.5050316029091736, 0.03298259967866791, -0.6479194094338325, -0.8017340699937345, -0.7997386373432843, -1.8351599017438402, -1.86890034980294, -1.8405329501237524, -2.7154739421090626, -2.299635678074549, -1.7446742396591135, -2.414306873745882],
[0.20461102726621536, -0.2815258562850416, -0.19228861204980235, -0.35785822759640185, 0.12550050073927593, 0.05832618885143773, 0.22690608313986388, 0.06062300598029569, -0.19287427420828882, -0.21184864147088203, -0.1474256554570372, 0.038877486076584186, -0.0519404293497886, 0.08860217668187424, -0.1401648813135771, -0.2426500323812793, -0.10646820160430713, -0.3952341621275716, -0.06490106307601926, -0.22767867912518874, -0.04927836450542389, -0.14729942454482195, -0.15209630475516311, -0.16952253962604708, -0.351348605850567, -0.07057470772081295, -0.22931605298232108, -0.28483572774728944, -0.6271905351661514, -0.5907803459154157, -0.27197971649615227, -0.43671252886174344, -0.6350051913330576, -0.7499791166427804, -0.8012161181917657, -2.3338843432445864, -2.029559957175166, -2.14229366203902, -2.3239365762415956, -2.3292952377456113, -2.0289462903019984, -2.682202225715142],
[0.13260149069659113, -0.19532894857942462, 0.3326409759064575, -0.344091779912579, -0.3043610096267086, -0.09716278965605168, 0.44987631937423656, -0.20217613665565815, -0.37009738686070326, 0.24080024333352779, -0.5152261955484762, -0.39204775037283934, -0.23196632160415148, 0.16081613750980364, -0.4450436533170348, -0.5061394288113177, 0.7514513597775654, -1.0094942963556555, -0.35692382388048705, -0.41643633607596253, -0.07536732970798471, -0.6090221961010663, -0.3461174671169418, 0.22757383672189085, -0.25164923235064895, -0.3397621002505583, -0.18323774436207063, -0.08422754983392498, -0.8751409613780331, -0.6268577450080329, -0.6650481308031592, 0.18440146361221016, -0.4743029282172724, -0.8005252227837467, -0.6793272397052468, -2.1532809819915606, -1.8368952673128323, -0.799764343662592, -3.2352644201121112, -1.484267340619942, -1.7640717982496807, -2.0535918151073362],
[0.016135750034300053, -0.22811196326214733, 0.19989054111542964, 0.40763043675304766, -0.0091082522299277, -0.22066015929769583, -0.039344901694600594, -0.06321078242261598, -0.3135030454257919, 0.12438731092111724, 0.11580023878318195, -0.11547478787849616, -0.10850890771832405, -0.18213379853616657, -0.09166838688511672, -0.04055032084802321, -0.2068642875498535, 0.08873111466990653, -0.20956488277226643, -0.13415897281994119, -0.16735968031966225, -0.2637771401026508, -0.4371125674852887, -0.1694091681800447, -0.20258101266887568, -0.20163746913033473, -0.38987840139887503, -0.16701255236124132, -0.41335891102692973, -0.11682084075119746, -0.30844700277584636, -0.5892342186368518, -0.4446176234862163, -0.23923989177824928, -0.4092596465161246, -1.3920637442525339, -1.1304750323340507, -0.9416459433555029, -1.189032430489802, -1.1286760131813929, -1.2279875772730326, -1.3334634091344664],
[8.53756497118109, 0.31223724662668073, 0.10927060749543416, -0.322520916298693, 3.208457159402126, -1.9951667759695597, -2.0867827671910444, 1.673224673769932, 1.1811911804010302, 0.846499388439438, -0.5823951788756955, 0.36746300509352237, -2.423888491697011, -0.9874208652166439, 0.6042859880981329, -2.035521557147888, -0.6173251223328549, -0.5360901043031533, 1.455358763593826, -0.9110503811551123, -0.4175316626649191, -3.268210940091702, 2.449139770920623, -0.30750442196745537, -1.4026732372962398, -1.2218926821589071, -1.839488152047383, 1.0978355010658578, -4.4924477184944545, -1.1417811358620045, -0.8101814860965459, -1.7317635136375729, -1.3147040142270359, 0.28737797612229343, -0.7657949869796411, -5.226781483869608, -2.699529167198122, -1.7875001775316408, -0.13097131714228413, -4.205566878581139, -1.8060944731001742, -2.1185521420308278],
[0.1394091965480264, -0.027877445505100542, -0.11408659625198003, -0.08753307963706533, -0.04826981390618166, 0.010781703077993582, 0.23198981092008097, -0.04982117143993989, -0.2075693202728036, -0.13545162600792826, -0.021574541243869476, -0.19939519953788418, -0.2005458595933264, 0.18520187820933634, -0.10140330619389022, 0.142679572670569, -0.191532909098846, -0.3111577300967517, -0.08186801594422162, -0.0014636413308007315, -0.20191592865168537, -0.24153255999198175, -0.404856976881936, -0.35261005831549486, -0.5262559201351464, -0.5493588483034174, -0.45778149836582166, -0.1875817634523521, -0.24759605212452732, -0.024572562351227033, -0.2581230590542439, -0.6801719398596057, 0.24326908075702322, -0.45865318883155304, -0.5485721910492156, -1.3607728916222253, -0.8003906713108931, -1.4977085919628146, -1.0172348450923698, -1.1933824595489293, -1.2871209725979778, -1.1073797329669328],
[0.11490070256939255, 0.26523692052056475, -0.016784000500582544, -0.2043216327862761, -0.1671443212401082, -0.05091343350464799, 0.0673227876996548, 0.16388692728635054, 0.1994264281352598, 0.22543837096621497, 0.04285887334876507, -0.10241277510762135, -0.11526527006526437, -0.07373606544281686, 0.051399291924203255, -0.26274255569837424, -0.11320656589394043, -0.057602741852505274, -0.2393549393156378, -0.282921135007824, -0.14456980132466657, -0.14027228723043061, -0.42438252458931053, -0.11159705692265441, -0.42568289656295727, -0.2171216706791732, -0.009205660901444245, -0.15685218252195784, -0.7591171671268665, -0.625608998078245, -0.6459949294671543, 0.023847329599996244, -0.43384395734531045, -0.3059859931723499, -0.3253402016942624, -1.8365801143457894, -1.3469999136838458, -1.0156016274193187, -2.4161176979160555, -1.0311122809288817, -0.7810192445553181, -0.9647434639436162],
[0.37192040951543065, -0.16887065670679446, -0.3548669601165717, -0.28526586435206147, 0.11433745914516494, -0.026073833680580968, -0.01864327016871069, 0.06377262476404896, -0.25319929134156377, -0.1578720298887042, -0.12263366808379235, 0.37875802150015236, 0.025011690094114337, 0.08665456256642493, -0.722320231025432, -0.4455905419844645, -0.3290557823923073, -0.22647464200185366, 0.12375085205723442, -0.3037992095929713, -0.08466549017472239, 0.5357207552275199, -0.8257641431388099, -0.1941335194334705, -0.07576912963730172, 0.048970955759443255, -0.4459584664408844, -0.17650854820921374, 1.1716093993970622, -0.7909869576517865, -0.773235522141335, -0.26172333251559504, -0.21684433583305326, -0.6321806790130563, -0.2785949365285234, -0.8913601017648529, -0.1978457238021529, -0.5294945282263527, -0.45964997971099925, -0.7374753797324055, 0.09102952097076066, -0.8890192273515697],
[0.11421660409070594, -0.4177968735097062, 0.1775680516557662, -0.37575470054525706, 0.0928504205450735, -0.13766008208936711, 0.15412807126420744, 0.01800517192064451, -0.368008754694799, -0.1529797514713959, -0.340597649262817, 0.07574386486049767, 0.05092740488550692, -0.09127854034210418, -0.03525402488902963, -0.19110401687912168, -0.10496910299162625, -0.2496731061429917, 0.10759849972964279, -0.2861759839599708, -0.08594285837990918, -0.2277541471156513, 0.11671863460263776, -0.015525275297852111, -0.2926513201850695, -0.0755248457293008, -0.29437023975936194, -0.17290946818801037, -0.5657276653390128, -0.033683398616204535, -0.5678456095088957, -0.11288395430162966, -0.5125060097607463, -0.5839818676740951, -0.5704168683231157, -0.936349114537157, -0.791432745136242, -0.9957387321540837, -2.1971046393478706, -1.1761200811415677, -0.9570733570814441, -1.6414856314631676],
[-0.13044728253695045, -0.179845361487185, -0.018906358341486483, -0.18339008443229687, 0.06233611312905931, -0.040418974486112105, -0.11974755797242, -0.045698328063172575, 0.050981030664981684, -0.046039692096161496, -0.07313048039636409, -0.12500253505643635, 0.18765932941293959, 0.04319492815625757, -0.4180998353604607, -0.26623849537850786, -0.2895729744463201, -0.5320304118162259, -0.1508934641672668, -0.2637045430636755, -0.07975399204495269, 0.31133537560489455, 0.03417335672718179, -0.2165742299661025, 0.11277472303149179, -0.40994547359393513, -0.24417368240427872, -0.07914228606719687, -0.1792460527471092, -0.4271187628279516, -0.4187153313875718, 0.03065141950718031, -0.5625288346972874, -0.3653924682452902, -0.3708077658026496, -0.16831505109236564, -0.7944128067808993, -1.016940602869222, -1.0409993135771232, -0.8116838418754421, -0.45665656126143306, -0.8135758151013113],
[-0.06244956210199553, 0.115422249777993, -0.019044186204078554, -0.18869613804331578, -0.12782209977645315, -0.005541571740132504, 0.11252685962573174, -0.02504625799300111, 0.006851714413665498, 0.03261344274195888, -0.18368456025797703, 0.037830165541884064, -0.213229974056299, -0.17120123132607207, -0.0758665576270131, -0.3290451179696417, -0.11826792092649234, -0.2535947319926329, -0.2793496180383504, -0.12851602261596046, 0.08763098948175302, -0.3532879724578511, -0.5388942532407067, -0.3647109645157114, -0.48613976474623805, -0.2799971647209045, -0.38124033138043956, -0.31677095656774845, -0.6513596452707427, -0.5267013409158797, -0.7255694261413026, -0.2961335372512104, -0.506547504000414, -0.4078932706268466, -0.7665582583282302, -2.1145267329072026, -1.5132475014353768, -1.6947437146100814, -1.8919805142423758, -1.7312160399486665, -1.4625981201816758, -1.6155480979503558],
[-0.08735164576176088, -0.041372982328254565, -0.02349088907638003, -0.22555667720945166, -0.07687030610436346, 0.1223387185458971, -0.0451894334312088, -0.11706814349443384, -0.09827832918256409, -0.2879094564654089, -0.2689699285463982, -0.12522477970701243, 0.12788814049256297, -0.08089896416475365, -7.44588380722133e-05, -0.37569816849463455, -0.08752173337414128, -0.35435228666850077, -0.3102509739429928, -0.31132059401401535, -0.22573539757746988, -0.41777886575891615, 0.4131136644823148, -0.0852492174132569, -0.06364586384219326, -0.24151055611326855, 0.12517498187849666, -0.09166254937843644, -0.5643067932675441, 0.36927151717339657, -0.6698057968063131, -0.5316669197351315, -0.43879704271348596, -0.40484807906791076, -0.48224927361506326, -1.4424752943552477, -0.38319854285393434, -2.089948075958306, -2.4587859736373683, -1.8952700404330765, -0.942422701210864, -1.0563121198242873],
[0.06064878587191199, 0.019628140326614304, 0.025045669288706295, -0.32723388894473127, -0.0886135716690281, -0.15968280798113316, 0.1254402304652517, -0.004994017865971543, 0.12327652553477698, -0.14831545977129615, -0.40282265493893543, -0.20435631849483585, 0.18419088555477955, -0.1362910642767175, -0.20194795997479228, -0.19954138792127443, 0.0008442442846668385, -0.6174362921462578, -0.16153072069495572, -0.44635156141798316, -0.12235038215036381, 0.06985791451357877, -0.21076999721169715, -0.23530534100422176, -0.3265373643756064, -0.22293356616793605, -0.1538552819566689, -0.10646941696980851, -0.3145639853587994, -0.5689308553207415, -0.2578029343095846, 0.23547381580450713, -0.425881309112323, -0.5481532982018593, -0.2590266732575932, -0.6581763714439364, -0.38731623300802087, -0.4708509857396819, -1.8625333009018976, -0.8977455187895916, 0.08740028573900051, -0.999401063740351],
[0.29499114623246503, -0.18403073120798824, 0.5876120523713879, 0.08839539362985578, -0.2431649296339296, -0.12947715072444818, 0.28741565560913335, -0.05031961774274682, -0.03585633814075824, 0.37080429749443156, -0.18860723079962924, -0.3055655320313513, -0.02865194937155661, 0.12656327347310994, -0.033510898338482004, 0.16400261426031165, 0.16708464028566514, -0.3216222884704797, 0.0321019428597406, -0.09118140034270573, -0.052463315084923585, -0.1524213093764529, -0.409713935888375, -0.15593192423783414, -0.325859442136877, -0.17596305485487543, -0.31919779190625325, -0.20520493641161047, -0.6582428922395274, -0.40189116475447106, -0.6343794045276872, -0.35624113629156495, -0.35935347027000075, -0.46261286241747374, -0.37828133430858457, -1.9684246586805227, -1.4242456746969765, -1.6378673911849284, -2.267019562976487, -1.4225740623872858, -1.566629785623548, -1.7020667883971643],
[-0.40030845122103026, 0.18032018284692367, 0.07891125925054839, -0.45231482762210695, 0.05729905251853611, -0.13957846115298328, -0.06712163263391457, -0.43306692124701573, 0.031879941239616114, -0.1949959615248984, -0.07385852964678175, -0.02658133893782976, -0.09093574075857858, -0.13238133563223967, -0.03627710818642891, -0.04421932555565326, 0.04106426120768896, -0.2965264796850494, -0.16313701778607842, -0.44741873462005494, -0.37153178173596474, -0.4485975928240027, -0.25163006514387354, -0.2252180517435106, -0.4882239443511581, -0.4266211014050815, -0.39416784728261234, -0.26948254474426697, -0.9832326915708998, -0.5573085968596109, -0.39109932162779376, 0.0398605573120845, -0.6568692079711729, -0.6161202500837579, -0.4709965665255562, -0.4818003119189236, -1.1595842050625498, -0.6501115590807585, -2.180707995104026, -1.1244689054972392, -1.1145706766037262, -1.0905871219383443],
[-0.07863908308718785, 0.1467682595280958, -0.2169904969238105, -0.0030566361530689856, -0.10309239343199052, -0.08247506340326777, -0.22501405538659994, 0.12002513342843812, -0.1969769489853166, -0.11128037897532647, -0.05759537799654757, -0.13847240491388285, 0.0034111989226422875, -0.01675145712789844, -0.14682500856681632, -0.1898134456003282, -0.2734893801282477, -0.23940058113199897, -0.31025559474268805, -0.18309480726269656, -0.08394941089638419, -0.1512097923238752, -0.30579839388617647, -0.11762229076116956, -0.16644067674776483, -0.1174639689025419, -0.43233268623756754, -0.35655068722609684, -0.5713735560622442, -0.4265500643761307, -0.3985682689057811, -0.4372645407655329, -0.36069575486640876, -0.3941341172427508, -0.6629668160457072, -1.3504371618251507, -1.3212995788222992, -1.4301432071395213, -1.932999494848756, -1.5169093926303876, -1.4522762312313604, -1.5708840683206808],
[-0.07435618172663801, -0.12674003673933465, 0.14563016181534372, -0.11072808536800029, -0.027805058403243064, 0.11841720028718646, 0.01976760587142272, -0.1554014277986272, 0.028465184231813403, 0.09295560933279728, 0.024945158526201418, 0.019989168607489543, -0.08127156769450819, -0.22370543738082274, 0.020486932537209467, -0.16464961946205425, -0.1292372611607077, -0.3435775633926828, -0.1861532635758224, -0.2728909048185864, 0.056545250863025615, -0.029594722852639192, -0.073254860547547, -0.008994544048159787, -0.36357244953411616, -0.0639044181479296, -0.021990979631067553, 0.011372222047451058, -0.3768086817626023, 0.012761548594164472, -0.3356398482011259, -0.14444863652873152, -0.08292960377230311, -0.16270661002182543, 0.08235225022033642, -0.30113325543097896, -0.12359295581406342, -0.19764122778142706, -1.4459946373473151, -0.1290451477635608, -0.263171044615742, -0.1691342503880348],
[0.8652083786379112, 0.3673148993317549, 3.9014550600426285, -0.4191993792590889, 0.39917528025390114, -0.8687104038302704, 0.5544001496167396, 0.9525706574200616, 0.3705576899012056, 1.894699550965792, -0.7295771203170491, -0.7276375223933241, -1.272053799863962, 0.26655766039017986, 1.30202760433772, -0.6792775675050161, 1.213445042705059, -1.0415875910557195, -0.5210265825589523, -0.7902666466018712, 0.24722070380184552, 0.6748837793571644, -1.1790846041163152, 1.9401784492187069, -0.840020880035236, 0.009464270678539449, -0.06800435619619678, 1.0242640103159082, -0.43407562817248224, 0.03525298101337937, 2.289736201093609, -0.21681564764652372, -0.04147747226691274, -0.03949617664830245, -0.007940186173078045, -0.7776541215417847, -0.7742394849649098, 0.2590135915430682, -1.6087618988787604, -0.8519607522268561, -0.15591322558128493, -0.9906305722446888],
[0.05296072394814966, -0.09332074743236443, 0.042757013088352676, -0.1892000300706115, -0.0018042701778092048, -0.11582970424394856, 0.028167569428684834, 0.0363534231461341, 0.02923289162474173, 0.030573227018262077, -0.11885063350460574, -0.060542567480919414, -0.14266476146381277, -0.18850300035822284, -0.11253562844146285, -0.21772958883412213, -0.1092242764375649, -0.41150164903499076, -0.2579538660728547, -0.14643885973501197, -0.0969029769656491, -0.3609063200203593, -0.318423045161813, -0.31239515518493477, -0.5823667612396197, -0.2890913046062791, -0.33343323574585426, -0.03524110003646493, -0.5932510148738857, -0.48358667318397197, -0.6299855621448498, -0.21964675452981167, -0.3131695612540018, -0.498897078489977, -0.44493938669646904, -1.6748270632195865, -1.1635867961021968, -0.962457009873988, -2.2065030173719773, -1.115365724072104, -1.2367332933989366, -1.2522930045283485],
[-0.028686128799443808, -0.26284196635364965, -0.09781449447253661, 0.0072155081302739555, 0.1966039031638762, 0.20471710756375128, -0.42132814727395806, -0.17940399331072732, -0.14046739592627133, -0.06623828515065294, 0.3495002113620386, 0.1444622092619083, 0.3724060255085732, -0.4061169755120127, -0.4933099681512922, -0.5345111842468144, 0.08214571856312132, 0.25210460183506256, 0.16438099966774214, 0.08940581790906164, 0.31192366370661356, 0.06452667805353877, -0.41584941813781723, 0.23517843090710022, 0.7382571339873248, 0.23906444470662658, 0.39846328366451605, 0.18852550204058582, -0.2984573782752299, -0.7191170538649678, 0.4916012020780693, 0.7616035156432279, -0.20768840612214248, 0.15221370362419653, 0.4361535880614217, -0.5444104466334029, -0.275804750339344, -0.17894120203390085, -1.0515060566796777, 0.3406819615823804, 0.2904954031505226, 0.5502261083846308],
[-0.005318671528714494, -0.031333363870930366, 0.20420892897053297, -0.28589105744854043, 0.06928320546906902, -0.10371222817391698, 0.12338868683095137, -0.06501850786701892, -0.09488208219326638, 0.16458954075874016, -0.22941521099928508, -0.06294516545918247, -0.04710468419661355, -0.09741629531932823, -0.06659885195932468, -0.10531338125127358, -0.030905724294924778, -0.01232235650277301, -0.02186355176031011, -0.1805030026848038, -0.14785861361624042, -0.1700037583894991, -0.3715858369421589, -0.07102661183893746, -0.41789327242253976, -0.1553594102667959, -0.2277195182001101, -0.16655536582557154, -0.7241081724616552, -0.49068520234289914, -0.6327897808332439, -0.22714902058234754, -0.5285745222379574, -0.4268018434321789, -0.8052506227363523, -2.923990646134155, -2.4952011681831765, -2.7866693986976228, -3.1081126691533862, -2.8406482961030024, -2.647967451651577, -2.8574210027229245],
[-0.3981717563226416, 0.3685193657676251, -0.480571381967457, -0.22955060384484136, 0.30071257541671287, -0.025075088463409837, 0.16249480566900332, -0.4961960751757056, -0.417478900926747, -0.5930237154215648, 0.6159664754659357, -0.11089682209169666, -0.3033139281018053, -0.05985123069259031, -0.4177834548605532, 0.8757966401198368, -0.0883903604611332, 0.7165200684343538, -0.08699242725039984, -0.1679275566269247, -0.09721886655519361, -0.7212777069466784, 0.4322996743130112, 0.3535535487528332, 0.6192073602562067, -0.46043198917707623, -0.30742881756227897, -0.2749058609575896, -2.448268842399481, -0.30487613003239405, -0.4734628352998046, -1.3913210561248748, -0.6049805388741785, -0.09556573125369547, -0.8195227670746874, 0.6950146162414509, -0.5694949833545493, -0.219980102883224, -1.4120663028190592, -2.2400172191567975, -1.5039581835390914, -1.8338246076724676],
[0.19116286932353968, 0.12619262363490166, -0.10485260224579016, -0.12455987990757945, -0.008101077634711677, -0.19090500964058935, -0.05393140289791774, -0.23969150817923746, 0.09785810828816807, 0.02177527267796902, -0.4708780781722123, -0.3420207547500391, 0.04719783886986438, -0.6210174925316517, 0.12468929326917318, -0.3124744489367378, -0.4636331827001416, -0.49081632348089127, -0.5755150587987035, -0.06641303766113896, 0.07206627305266428, -0.29625837839238495, -0.2372788064600285, -0.35562215696504096, -0.3724817067433306, -0.3903601442180594, 0.2544943369014732, -0.09534362737474336, -0.48493971324771884, -0.4773609673845931, -0.8479935857920491, -0.037502801714295785, -1.036427541743313, -0.5427847673312695, -1.2113959134029046, -2.507437267267968, -1.9698308436363652, -2.373477279059096, -2.483716186554172, -2.4741741601204112, -1.9833681427642442, -1.744069486709486],
[0.1860700904819229, -0.5148798912643733, 0.3240293158208753, -0.05194047107529866, -0.5018462759686895, -0.42677939137339604, 0.011110676655836843, 0.024493375459702982, -0.3851183343395588, -0.025200963249246568, -0.33473989236020785, -0.2638806970664517, -0.5987074716758515, -0.2914748691309961, -0.2411871420674322, 0.179071669312947, 0.22461615295141457, -0.32612298293650693, -0.4090630529620934, -0.18890319950673196, -0.3833680081292266, -0.5063508522779299, -0.8034031443551388, 0.11284015545005603, -0.976755028863473, -0.8158131887632036, -0.9450028418492029, -0.4131148607491483, -0.8110676273940743, -0.2112244811913227, -0.6571494106814075, 0.01136971653331623, -0.5364850435326751, -0.6269763776133794, -0.6073881548379922, -2.9754349026756413, -2.4731990727300732, -2.8039519287645605, -3.2198637350524266, -2.7518879437361514, -2.5017268863221394, -2.8899619453690484],
[-0.11510322339128372, -0.14909832741560053, 0.1582603836878403, -0.18032955355628005, -0.21014181326449805, -0.08959174215112978, 0.02545277762033458, -0.03345172434852728, -0.1769196454269853, 0.21239822115878093, -0.3296793670787053, -0.016107181517595184, -0.24832253812068886, -0.06263738479651863, 0.14536703223883846, -0.010428941200330914, -0.020040843957415958, -0.3656348008633814, -0.2786908647299206, -0.23352642067474597, -0.17660331897280315, -0.32298119666146763, -0.49570579323085007, -0.05729898382713653, -0.6138657050829003, -0.34214819420009623, -0.6223349536393649, -0.17801163790452137, -0.8767190902628903, -0.1788485910831325, -0.4772596593303975, -0.344596845011776, -0.5241541522440413, -0.362068592047019, -0.7402353865075747, -2.4303544442909493, -2.0110325689016717, -2.410631483540231, -2.6158165224718632, -2.415485404984572, -2.231430082846974, -2.2708685920513507],
[0.10111946060233235, 0.07857294107721093, 0.02589041438216929, -0.2919260679239225, -0.21505043910645488, -0.09177676432045775, -0.2960474641195631, 0.10903430999907619, -0.2902427233421892, -0.09188738667564313, -0.33258406505280985, -0.31670687604846354, -0.34540504794382526, -0.3504716488422476, -0.17484598548859717, -0.42461326004499556, -0.38560107178250796, -0.5300546600143111, -0.5484463705749097, -0.37773155463617925, -0.44284402091762376, -0.2523019589947884, -0.5811071796823292, -0.5951101244356415, -0.5294981075642233, -0.04226873484878528, -0.6067983219663253, -0.2699945217192873, -0.5516328802012982, -0.9393942579039529, -0.6610905764059075, -0.19247713525821433, 0.14056429322836933, -0.2587511321762119, -0.6555851365378689, -1.20634535152981, -0.24307750076288778, -0.687786947978666, -1.187774515466517, -0.3393054268125391, -0.3733532334509274, -0.6290623300316757],
[-0.0863031289219767, -0.1296988322101729, -0.14909220622260264, -0.48822576826707437, -0.5019478716457685, 0.13999672069501123, 0.16261715135078497, -0.02615158265328041, -0.1699973829826997, -0.2571884662869588, -0.4508348362030526, -0.3262132503702284, 0.06687275129384576, -0.18156133869530675, 0.10546985652136308, -0.10487302663137635, -0.2301109921213108, -0.526920616516891, -0.5171803796150142, -0.2803690474273158, -0.4082626742217746, -0.3464386387816313, -0.40318035634611943, 0.18607429619369203, -0.7119968290103357, -0.534819238086545, -0.1655558851736911, -0.32956517271047536, -0.531715435818391, -0.7006835421526851, -0.3336742075556806, -0.11967262296567319, -0.49618186580027235, -0.9051503286458932, -0.8000951638762178, -2.439450769375415, -1.9087164926105433, -2.35677861495943, -2.5791879156434097, -2.4967193264982757, -1.8657375869135073, -2.271186565446818],
[0.2808295103830068, 0.07843280036218526, -0.06383654984400904, 0.06672707853745527, -0.23978264212489925, 0.01405256268185088, 0.3070727322403454, -0.017541502155847936, -0.21694004398308583, -0.27418113031933306, 0.26181126788582815, -0.2971220273154484, -0.12969318812542513, 0.1578419235099696, -0.23116475264805386, -0.44159785263274376, -0.13065097232016717, 0.626602230915117, -0.18777360596764653, -0.2791924628455246, -0.04911814970699579, -0.1570396045305885, -0.6170273476422767, -0.21353300063282815, 0.4189423474524112, -0.3759661086654898, -0.5029338161640092, -0.20418345974593893, -0.4213703243179686, -0.5877994451313961, -0.25773087786557675, 0.19245676316200383, 0.4230483078736585, -0.5641754334655253, -0.4402590642857703, -1.5012775210388665, -0.5213808660212977, -0.0987425693753763, -0.1102805844158345, -0.16566247320440797, 0.44450758582587147, -0.9448084521525452],
[-0.10700480794422218, 0.14819839038637764, -0.1863732665114308, -0.3138830498338447, -0.151394487511575, -0.11308806066462125, 0.09534826507037618, -0.16851944617743708, 0.19934191246145422, -0.14369500141659486, -0.33067365564195783, -0.20172462117949197, -0.15189551097129259, -0.12438076951375009, -0.15426963780956332, -0.1397902200055874, -0.38901439615363176, -0.37934975787944236, -0.040775145930597646, 0.1712964783896083, 0.16093638633874757, -0.15575981959728388, -0.2176560027666103, -0.4310619036157701, -0.5760578464818027, 0.1934833109594958, -0.10532770989012379, -0.057612996865551595, -0.1987095326490357, -0.4618649684594639, -0.6197581478095272, -0.2843504694340057, 0.10291541723626983, -0.4201160531733288, -0.02072449964057813, -0.8055190985077494, -0.5858127311913846, -0.6430216505981114, -0.9185026810945454, -0.31887082644989306, -0.8332475892369051, -0.7964055422105026],
[0.19800597230057732, 0.1259897050908087, -0.7979468695534654, 0.006060303962009002, 0.42917276377824976, 0.5840620529386369, 0.5556444266725357, 0.21706346778835717, -0.08473813514654129, -0.8349981270656919, -0.04969709627807905, 0.21656910640680951, 0.2956072751677609, 0.24611866372623856, -0.13861964404564622, -0.337412212251195, -0.33507063480984794, -0.09621519561346553, -0.11918418051844425, 0.11565992290556071, -0.01844171928963633, 0.0987879477681215, -0.35180733994601343, -0.5966829672552637, -0.27946507257370484, -0.1879916505955943, -0.4721033993248887, -0.2520825641775555, 0.45653213984225993, -0.7625193421172434, 0.2569884180406358, -0.47389338129619657, -0.32247456891460563, -0.6763314236565902, -0.6936158857172005, -1.3571566450237509, -1.2178633661538893, -1.5165855623975915, -1.1410170333498573, -1.887231474917109, -1.528635558604665, -2.162937847488545],
[0.20071628314550785, -0.06410619183105828, -0.34011764060743827, -0.2544788246744183, 0.16233204722362413, -0.08863123373097366, 0.16227630439944146, -0.1657498547925421, 0.10150057796293892, -0.3519280341486416, -0.5235201409357416, 0.10436232171990652, -0.42042769543107616, -0.04097806379478923, -0.40426047012205824, -0.5046354130447652, 0.05335191881664649, -0.4360055864053141, 0.3885758057213091, -0.10126743737022414, -0.250651581520277, -0.8434506410455603, -0.6646445608266106, -0.7363784256694043, -1.0555283881711806, 0.566759166854329, -0.8364386850692593, -0.41234026904095405, -0.793122465350485, -0.6773364228921626, -0.18843878215562443, -0.4276151603299408, 0.17930203713783135, -0.5947519255757072, -0.7378724231058951, -1.6413941299453552, -1.500330956795616, -1.872439094460462, -1.610321395818802, -1.4724158508754308, -1.7972463417680276, -1.9779683437596745],
[-0.03533723492508858, -0.13119815324609302, -0.08859301243576012, 0.10151665581489197, -0.0676491176144067, -0.05507318691773872, -0.1012233831370784, 0.06650894748812518, -0.09976017416898515, -0.19444366559695073, -0.0627818255224269, 0.017630931761339112, -0.012802579329073915, -0.1394844804925385, -0.18073334140738626, -0.19888085789474588, -0.21110856210056783, -0.029648077773376986, -0.15327697908088633, -0.11941606893558132, 0.009989224886499544, -0.23929368975625473, -0.3332382642054704, -0.05465691364889476, -0.22194056687396493, -0.15162789547073172, -0.23882352272868232, -0.25117645502752783, -0.4610908124921476, -0.26459358430140273, -0.3266368638622455, -0.3987066145611959, -0.3428681785337549, -0.4469344649654098, -0.5362025162636775, -0.6482185170630649, -0.9693740117357291, -0.8829609399539199, -1.530204260391933, -1.299025501242484, -1.2708099413177523, -1.2735902024889922],
[-0.26024518385265316, -0.4637254166730605, -0.14088465413539966, 0.36533995079665926, 0.958059462060346, 0.32071143192269397, -0.17660628382175703, -0.28701262809848155, -0.2588959155654427, -0.5853276228540842, -0.08270154883331839, 0.3343362385563436, 0.0099159447594476, -0.16888751148254896, -0.24645236403319057, -0.5341599410619859, -0.16696936416359284, -0.32976409883066427, 0.5558883553119692, -0.13349050723277128, 0.14618644451385376, -0.05959832607108988, -0.07568085253736585, -0.07724843197513505, 0.018533581301581403, 0.12320460589056607, -0.03742315754316011, -0.176425851641081, -0.5582708880705417, -0.29949963060594265, -0.5704228768496307, -0.1764798883011881, -0.6741841763924934, -0.5599304861013003, -0.7657497576795976, -2.6262880870170644, -2.422207434374516, -2.634125956327889, -2.8974456806457276, -3.1099802308008675, -2.5991285169427156, -3.032565358067744],
[-0.28851689715626677, 0.1956287071474222, -0.1764265423937579, -0.6504763605730559, 0.15305991466808533, 0.15175239873955962, 0.09111556095735437, -0.17192932294294103, 0.02133849974780785, -0.1693110828200393, -0.33511272790840035, 0.10644368748040163, 0.011396688650194584, -0.01908585586787095, 0.058892058749707904, -0.1727716593979503, -0.07878353750112554, -0.8704831031863236, 0.029165188664784653, -0.41961337909208873, -0.19907824049638978, -0.18626740048144186, 0.15670020697422954, -0.3191617998510219, -0.47069824641157404, 0.14487145870657062, -0.020155265077861383, -0.159285397729435, -0.4459738345336588, -0.4841346210191857, -0.36020698468774665, 0.2177454940329571, -0.4416470282999862, -0.5727988143119114, -0.6825160829450184, -0.9854193150023862, -1.0684687583248995, -1.2058657797617411, -1.6406156331183421, -1.3749368254864227, -1.0497045157321445, -1.8308364184878583],
[1.2393974894560242, 3.518834446394123, 3.8029612729961153, 0.9256753799880064, -7.485885833220319, 4.284470917475509, 4.741227160583766, 1.2432511465224199, 1.7155077137200414, 2.669596654061437, -1.324087956660944, -0.9997575026098519, 1.7779483833632985, 3.310829853532772, 1.2527364910120424, 1.0509580179626854, -0.12512104397999313, -1.7797617420228213, 0.4896539630400418, 1.8847866766493802, -0.34773923816881924, -1.272358975425059, 0.3529563004624269, -0.3089301598833767, -0.7371442024155461, -2.76629207531091, -0.8446684508731664, -1.161557523686972, 0.3671399817100259, 0.8990593296959029, -1.4791552750667478, -0.3145595784409503, 4.847249137572438, -4.177652341744809, -1.3128251271145306, -1.916655002941844, -1.1570120910780615, -3.122622622040341, -4.296571127264785, -2.0747031090517773, -3.380167208005112, 0.31576952337311404],
[0.06227208235218862, 0.04874615679951564, 0.22422806605623882, -0.02686499239870511, 0.254662473012496, -0.33308686219107875, -0.09471548654792206, 0.09637563653060777, 0.15817802463122801, 0.16378816851965558, 0.060567324488193346, 0.04421652061786829, -0.2203383686580011, 0.008333080011324873, -0.13332396442304323, -0.21407725888333637, -0.03259116956158543, -0.18695529040247924, -0.209090829425221, -0.24823590211080868, 0.01493486683190374, -0.11698830072383727, -0.5137338119955749, -0.043882966984844465, -0.2121647626424775, -0.22447949294033356, -0.5603457440707654, -0.45335178704976986, -0.6058925976698524, -0.3986174927903405, -0.4538460614644954, -0.08496584862144253, -0.6046862322597035, -0.20664150545240575, -0.4626662077885925, -1.7647389885414957, -1.394209265350398, -1.699032800810831, -2.106805128317671, -1.471280517672026, -1.2379027706844499, -1.6359916268651935],
[-0.03304676656629049, 1.1708002902691, -0.47307117422429634, 0.17670630138468837, 0.5884259265343768, 0.14021016550253967, -0.20911702832180076, 0.3095398835077875, 0.5953600898378211, -0.5870800130619829, 0.6179191702335705, 0.07802384331239502, -0.0035968970512950207, -0.4032148623406705, 0.05063285144519957, -0.007701051201323674, -0.5359058738125574, 0.18503867968194118, -0.21222676974761154, 0.006505402239377637, 0.14566224537547354, -0.023729478482597852, -0.6257308265347695, -0.2889458393657928, -0.19308125694034572, -0.5264350201662307, 0.05454043432040118, -0.24224331988529424, -0.8106811637933157, -0.9548639253141015, -0.5708200551397494, -0.6472371966298894, -0.3050196990365028, -0.07382727644955459, -0.998617638915904, -2.1956709444003355, -1.9703825265042243, -2.635100279763246, -2.1529911728547915, -2.0865267291473506, -1.8540622572856558, -1.1361473389360222],
[-0.2738604563558652, -0.016916186455486322, -0.20546770306621828, -0.4611526563724922, -0.0579688721047754, -0.09485016465417226, 0.008119131521206331, -0.34986966081358406, -0.1195349501570087, -0.4389480763595015, -0.2672927675716778, 0.1301658266211329, -0.26345603171970455, 0.252877224404043, -0.030779542095665354, -0.37855339464403465, 0.0851000810562137, -0.4632902358894307, -0.23709845156627965, -0.2294700883142728, -0.03256106574004215, -0.3068301742298843, -0.33527667752227114, -0.2746887750161401, -0.631038111380561, 0.18524558884965514, -0.36308969124336704, -0.2455089771070507, -0.4698265879164612, -0.7541798628470767, -0.3542475621683448, -0.1633749038331823, -0.20370980743202893, -0.6610083163265783, -0.4128333199209691, -1.3571761288761728, -0.9196633824495342, -1.3801853739427439, -2.073553188125394, -1.042524654325113, -1.3390757721714708, -1.602830852940107],
[0.33297964621238185, -0.32018034708330695, -0.11439842850308225, -0.2882171397722255, -0.07966992642791196, 0.15008921196121194, -0.0200734552005843, 0.13119535220511264, -0.31173220493747855, -0.2946617689912261, -0.1985834950784237, -0.24620414325577425, -0.025856662569763333, -0.029459212715609407, -0.16109199375682207, -0.43741391179613576, -0.02538715692688404, -0.3243355053250165, -0.10217840569195852, -0.2816285062741253, -0.3414839845482757, -0.4724373842092503, -0.5269751039835985, -0.4295640096503834, -0.19091408634964205, 0.0038198724993131145, -0.19904311783955933, -0.46801794923627305, -0.4407158768608158, -0.6776421021803586, -0.243883832144714, -0.5695984448247011, -0.35061892730534056, -0.3378216143307713, -0.6669904226245521, -1.9574357608796338, -1.0168951728094477, -1.1662021189840155, -1.605551950022832, -1.1380813839337103, -0.6265179426335528, -1.3564551898935804],
[0.05594811813285523, -0.02127526886171073, 0.03525074965982851, 0.1526611952130982, -0.047999408616726566, -0.14331136816489173, 0.23397994861747093, -0.030127055526838854, -0.2613545379303988, 0.03674957583532092, -0.1251862672340588, -0.12733750099652844, -0.27367359787208745, 0.24744582425703837, -0.17851464671922443, 0.17252032198831255, -0.06806507262712667, -0.2187008540987264, -0.1367345702719837, -0.2501910173435984, 0.032412784747538, -0.24631446399332513, -0.24810893574882992, -0.1696343320024336, -0.3652636744130739, -0.11933008149020995, -0.6162529414970316, -0.24001809841123545, -0.6417351537167362, -0.3399459023450131, -0.5588422591314857, -0.5991313118327929, -0.19733627657296968, -0.29185481738602626, -0.5122573100024823, -1.7184829621471671, -1.3777811838966196, -1.908426015725371, -2.0444706125777814, -1.5130563565990938, -1.5970530720148606, -2.071350644617091],
[-0.10381847055654307, 0.2388837268486657, 0.09902946798630774, -0.04555302357620782, -0.04759384834534187, 0.03189680936554056, -0.4044649260557845, 0.05644530465314712, 0.07578021645602397, 0.043325668609721896, 0.11447520743652712, 0.05758962811330872, -0.15714523922724113, -0.2050522998156925, -0.24503889115856348, -0.04046939706795153, -0.15712247638536284, 0.03711122594150079, 0.02211325358745923, 0.013133481792744987, -0.3481619426773105, -0.13781655440528917, -0.17031541997799582, -0.3284216322242034, -0.29378916282344464, -0.1719900080693224, -0.3355601620801479, -0.16607241400852482, -0.7391713992955047, -0.37352198125484254, -0.48190290550870457, -0.5806107073187285, -0.4795287563060278, -0.07752715563637014, -0.5028379390016388, -1.5467590727726626, -1.3843152914591716, -1.757963611142912, -1.9289586963640535, -1.5335672965224363, -1.4004342821442426, -1.2979867649403218],
[-0.18374359513713054, -0.10109566787810902, -0.08195048829530846, -0.08494784603334866, -0.1757289119085066, -0.02822718377342071, -0.00495049243964253, -0.271243666914066, -0.813826938477108, 1.4125850316591142, -0.7041105766501582, -0.16713345027205376, -0.5953635211136671, -0.28312287599334285, 0.020696390384712514, -1.9208808911853996, -1.618792759490609, -0.1736939060211863, -0.41967072522026855, -0.22731106924160444, -0.48487048028050683, -1.0806570924984347, 0.9316984831132369, -0.19190210429582813, -1.6445262193421122, -1.0923495610976826, -0.613534816672421, -1.0072996951942932, -1.0403160135504135, 0.6469553792127333, 1.403991224768764, -0.05444136132829441, -2.7099950160795774, 1.239627871278403, 0.15474702163290388, -2.4036221888376317, 0.65273973080302, -0.626112566155969, -0.4430824513811535, 0.2103256309569489, -2.660732050271663, -1.5761831949211929],
[0.1410747643455052, -0.13246134660171763, -0.06995709666010644, -0.09614002215856204, -0.0023530626613146176, -0.08363447211294817, -0.26792236514531764, 0.003501169833649598, -0.05347651046309142, -0.11187710082023063, -0.0739886502053116, 0.1545277242734535, -0.17941899073728232, -0.22696182982718996, 0.14580349170639295, -0.19308092009627414, -0.018974170757265048, -0.21431559490991361, 0.05318092140393214, -0.1442677289637418, -0.24720814848873876, -0.06957080449286979, -0.5320688762266033, -0.17282760033360856, -0.2117745747674313, -0.2754362135326938, -0.18521085468211934, -0.2580184692209652, -0.6725399400866728, -0.3341715542881886, -0.44555572463097054, -0.3912444530388909, -0.5262164026658759, -0.24206333563402146, -0.54159736746253, -2.154475794678631, -1.615966715365632, -1.395753560081773, -2.1071276013994322, -1.5013244324036976, -1.5265512074727414, -1.387600252239348],
[0.8643640352171398, -0.3600384820211646, 1.2240299462914386, -1.1980569866816184, -0.42092166820747706, 0.7367917696067798, 0.2073928747914458, 0.5321652390438428, 0.006854029596239469, 0.8935363385843991, -1.5445316951269894, -1.6757885596225557, 0.3824609545180921, -0.4836215101287356, 0.3530705414907401, -0.08152681642879082, 1.1040769492116613, -1.743994922036886, -2.6225115533616647, -0.9368590300442373, -0.8923617696058788, 1.2501915576411566, 0.11750343139892863, 0.47306866417884624, -1.6763421573780453, -0.7465915842322222, 1.5363569008387017, 0.7464374082586697, -0.47349140139364454, -0.16151027533002849, 2.3852044426022685, -1.3220941730085327, 1.4690200003543044, -0.9789038684745288, 0.4234304892449655, -0.1599440749687086, -1.268479062784887, 1.5910788580205868, 0.15195556909267025, -1.36235778561697, -2.136477761518854, -1.7816839356766272],
[-0.18439170321990353, -0.9004519554278119, -0.8354496315681269, 0.34219944612925374, -0.38991558269318427, -0.0024605291197461487, 2.3029901744726144, -0.15928715249963507, -0.486062098608312, -0.4121775467089285, 0.0729429092881975, 0.21517638535968825, 0.2906734583613169, 1.6713327445740984, -0.09818307544165909, -0.10715841491097676, -0.09371626766636167, 0.0973281138590391, -0.2144339952113254, 0.18763986214657485, 0.38044016128961716, -0.35449142920337257, -0.5940763221143758, -0.6204153898996579, -0.22610458112376366, 0.04341794510261547, -0.20920996616337267, -0.5597562125597432, -0.5853370316127268, -0.4381261282028894, 0.471637095819228, -0.742470394383044, -0.5499325754226191, -0.7716186335310136, -0.9834869187387554, -2.8229623251035343, -2.6350624814746952, -2.8968478539582407, -2.590042078295891, -3.0579477268571282, -3.0955473777728315, -3.437528119510005],
[0.16530973526724865, 0.07928977416662533, 0.21846341003386432, -0.23962353970138495, -0.11512199309822525, -0.05181521067580348, -0.1595451415109181, 0.1267686508418378, -0.2169790422025638, 0.1940760453826962, -0.2857311330563585, -0.09126205979321068, 0.1276768054018971, -0.25127466483271493, -0.1850995713248127, -0.3480862730851026, -0.10153473280555221, -0.06574471575040239, 0.09168015951000556, -0.1336850956715145, -0.2538324929338795, -0.2354437896231831, -0.24369027556340803, -0.10766567682500566, -0.38354285189638526, -0.3415787081120575, -0.25032326082884093, -0.2152363742752068, -0.8119391646979818, -0.5800453869149634, -0.6527301314721135, -0.18280402291420864, -0.5127880318872245, -0.4497905726162449, -0.5794471248644978, -1.975719979359944, -1.5599519391957912, -1.631448608788614, -2.502057163745786, -1.7924510621314627, -1.538239221265328, -1.4475404421340345],
[0.2614386603596654, 0.36838658175885897, 0.07905458238668402, 0.2497844753980247, -0.8370496050579282, 0.28035859510533084, 0.17627162706696892, 0.23075573151463638, 0.06511185859062107, 0.2949827170192809, -0.02414228104749805, -0.5105956843762832, 0.1913969568853513, 0.2250273557355157, 0.337007749344338, -0.09943656293252409, -0.056779865464901304, -0.13382358634930955, -0.8931107326146647, 0.10426205399550137, -0.19981691316332434, -0.3652266809164463, -0.19101804313726656, 0.05533077228592653, -0.3649833092763628, -1.089776614327277, -0.5209503320930318, -0.1791756753593894, -0.5447604270688269, -0.4924396888872646, -0.7583810199038539, 0.09104381355008283, 0.09958148186406895, -1.0071966481145838, -0.6966188744041291, -2.549103829600905, -2.032837663830911, -2.46917648942092, -2.4903183737746986, -2.5128439704684125, -2.194244076854122, -2.036699755722562],
[0.06863423075178078, 0.2860035234324629, 0.020167552918377872, 0.11380955674829689, 0.05902005552698022, 0.19692804164533317, -0.27085406641753623, -0.1705407702990997, -0.07964403921457427, -0.3763002702079475, 0.033669701849428885, -0.028567097766436008, 0.0005572172397181616, -0.03171546853392701, -0.1835949345534627, 0.06956155027827174, -0.22670924093207173, 0.12622318662298226, -0.12038008825845835, 0.06669708013640924, -0.14634574762846184, -0.3145113113633679, -0.3007226393896297, -0.5472295061083459, -0.19586117733675926, -0.533083418006486, -0.49692341670740225, -0.004837956104732551, -0.6871511434430957, -0.5243874562309874, -0.5330734452094327, -0.7825359734010277, -0.3618690118495479, -0.34699794694752517, -0.6259639507427599, -1.47800348257917, -1.8736420125970281, -2.1513947953593893, -1.6272941985945257, -2.1477522466135044, -1.940679623198617, -1.7651455778264717],
[-0.11640222677453227, 0.15802122794759754, 0.13739204520592666, 0.07361115791044509, 0.04031604950789751, -0.2033200894756315, -0.08865045405705248, -0.2482092497629083, 0.09919340008832851, -0.2555598405637318, -0.16181411468774792, -0.2927412074185025, 0.001658538116390468, -0.17347899766307515, -0.22407399426533778, -0.011595895609964747, -0.2823990475667897, -0.3816393595163066, -0.04569089047330129, -0.2020557650551103, -0.21673229607036565, -0.45320299243558226, -0.2415798232330097, -0.22725948530675175, -0.3993854163889587, -0.3434127557984206, -0.23706127624176526, -0.2685276184214343, -0.764138878451488, -0.31507685823316217, -0.46318825850696627, -0.23637672735869156, -0.4562109467105129, -0.19234156981210895, -0.6579171757064981, -1.4653082800417998, -0.9640477672377478, -1.162783821218712, -1.3011865122712654, -1.3351629468642408, -0.8466807385105486, -1.255735780719928],
[-0.019948796928192195, -0.14746936867635277, 0.028307979650050753, -0.11137168492186542, -0.061702343055717736, -0.0665905430874241, 0.011912603622358197, 0.0351047534821058, -0.26001465431047655, -0.06119591146520016, -0.3985565505043243, -0.05724394436187046, 0.22849209432339984, -0.13956800670299968, -0.021912461103112495, -0.23718973208044225, -0.16149377713034613, -0.5424764385501627, -0.27965572010397427, -0.40805596900718516, -0.15962690938800003, -0.3357196484004539, -0.4695667162039442, -0.00920955392023498, -0.5203793075337598, -0.13252595437848633, -0.11028358899712382, -0.24704457137775843, -0.4919863681460768, -0.6958028594002452, -0.472976370523722, -0.2150635534257665, -0.7152628805711118, -0.4815094334975825, -0.640110678222925, -1.6627637857386905, -1.2947604153486678, -1.210739786284861, -1.7519046691789373, -1.4880100153278548, -0.8876140827861495, -1.8982269129554523],
[0.754537085821547, 0.03575857200492936, -0.4539543839971823, -0.2548307410854819, 0.7147471451383164, 0.10565245543787764, -0.3755075773214604, 0.4580578584059736, 0.15997132815998738, -0.4180194393325159, -0.22422762860178025, 0.4962235573687877, -0.10051012430200842, -0.3283578546024078, 0.1761422530128764, -0.037939423070345736, -0.041154939274133794, -0.2766790847160604, 0.16444650569668354, -0.11525288624578128, -0.37652611902899896, -0.42686033208469826, -0.4869272201225364, -0.380825473830869, -0.39998585273528175, -0.23656590489675738, 0.08580404623646436, -0.23838048484486185, -0.6588910194540452, -0.47075149550824374, -0.21970578823594764, -0.0971936022726345, -0.6874565864852782, -0.25695145125579133, -0.6505417397676644, -2.7494094172405052, -2.3680911613662894, -2.134990065129185, -2.654856404390756, -2.709160927931662, -2.059342811713367, -2.6159624083556006],
[0.4515215006100923, 1.5502592079685904, 0.40841711484878374, -0.5114690360562159, 0.997339440318008, -1.5474396958677297, 0.7539681343236246, -0.17700475729028836, 0.7514012816133689, -0.29176293945203163, -0.852266175395108, 0.6105257978923915, -1.0787811006980006, -0.017231400691844583, -0.09059854454095362, 0.28528793634122, 0.07521626709868087, -0.974984790077636, -0.08924513694166415, -0.4228323710820028, 0.15773708628698166, 0.24116712356774742, -0.2070339089800332, -0.8781484880996309, -0.646226244550201, -0.36173340634710843, -0.25034887944191586, -0.33901510885325414, -0.8683417614689304, -0.8674186529839573, -0.6563755572770433, -0.3840479782861537, -0.6530717787510419, -0.12456228185716477, -0.8481586495201293, -3.5676997696820036, -3.26212378604153, -3.7859569952659093, -3.4593342799854803, -3.6931426520000294, -3.415666225687162, -3.7400044491438207],
[-0.05580296474070947, 0.0023441844597687817, -0.16701358656441392, -0.26805012581935245, 0.026291313868409674, -0.10072643361534223, 0.05719624513217373, -0.19065825168476355, 0.053415570784507795, -0.01838393054006524, -0.1408044683679972, -0.12041131224896623, -0.15140512087619784, -0.01485426954327447, -0.024817372025891262, -0.2695741776751163, -0.03838838605420761, -0.43234369623229807, -0.25663942722731403, -0.26526355711147714, -0.25279437208888167, -0.29273873796426303, -0.18702674726113283, -0.44009455249908547, -0.6422614728275834, 0.01381256643102657, -0.17282525438747506, -0.06750157021310453, -0.49595415386390973, -0.6651884212117948, -0.5409985464682537, -0.12702461162876535, -0.33988516971081706, -0.4477896491797086, -0.6114881814804632, -1.6008473874709634, -1.094519624245888, -0.9031979222659706, -2.073183165955126, -1.170850300655912, -0.9909327532222417, -1.486876897186254],
[0.11704620235452247, 1.3376540292588062, 0.6957148566511331, 1.041043342679283, -0.2625451841168329, 1.7812713048483184, 1.2756330723033362, 0.2171341574403384, 0.20566070191665, -0.16023232779005692, -0.4317532311648718, -0.4552447970591592, 1.4260624127815262, 0.25605682435265686, -0.5126511636112558, -0.5307527259003847, -0.4303889270849697, -0.9704486627218253, -0.49687473585378616, 0.21102310537201552, -0.2260847250408605, -0.32541530428878984, -1.0288949244022216, -1.0943623639883064, -0.8480146510946489, -0.36447667540148154, 0.34215651425414806, -0.4740071822963333, -0.6754829342804779, -0.930542942101383, -0.6483412333810801, -0.8285953643867094, -0.9775443724238558, -1.0181083247572495, -0.8692441521502365, -4.743998515599902, -4.273591877667759, -4.644737216853793, -4.547869324300984, -4.668487042925176, -4.390431763235343, -4.671113996893115],
[-0.01654418114793864, -0.1171677331706377, 0.06653058089837091, 0.03788700281367767, -0.08358430916647944, 0.01302557121752726, 0.08483090281443041, 0.020999318475503802, -0.4283589742513071, -0.09106022990820381, -0.2568885223170334, -0.19045949621474215, -0.4852268180065971, -0.17272587051116886, -0.25407768188390895, -0.5206242846819846, -0.4472968486891152, -0.11271644139488425, -0.47560898575814176, -0.21398667865280488, -0.28953770362691594, 0.06141907615873489, -1.191023292115089, -0.4009353078425254, -0.36317211266719057, -0.3650697121507878, -0.7545058893670519, -0.7309715209623814, -0.4861633759983443, -0.6849423412420473, -0.3609327493992855, -0.5494275042396051, -0.27140718867769736, -0.6375737307139577, -0.8662072925827587, -2.654409172140162, -1.9149038840098533, -2.3136225605736778, -2.3923133800911582, -2.3147172686693263, -2.075591361480875, -2.6171825343044572],
[-0.19427876187157078, -0.023748085664869616, -0.2508527312364344, -0.05699636827338764, -0.1476385811027855, -0.07069258354385415, -0.10044406337273061, -0.04740605656203147, -0.2020154724997904, -0.2760987270392615, -0.18371700017976167, 0.015819524945484022, -0.04326840055026153, -0.09924401015886598, -0.2990440210350213, -0.34338948448862316, -0.210336411417744, -0.05088482209190142, -0.2776538607000616, -0.14331947796986624, -0.11126621455485847, -0.1564671576641698, -0.14289215933346244, -0.46924898129766346, -0.2377182357374987, -0.36120972024498216, -0.1640783144911278, -0.3280971521671031, -0.05385042005452132, -0.6028164989626865, -0.28750884671833354, -0.25311795511226604, -0.2973184671220203, -0.603685180520136, -0.3413658698190187, -0.9987890230807104, -1.102307875188048, -1.012347289768808, -1.1748275179448404, -0.8644658179359341, -0.9832403454864114, -0.8379331028460345],
[0.2643703394061303, -0.5079429003293918, -0.404368373509345, -0.548801041341807, -0.4689610822556658, -0.3057680788721099, 0.5282129510028829, -0.1988024824904608, -0.3369146360515528, -0.6918382384511799, -0.6815217494062422, -0.3443078060302295, -0.2841096274705609, 0.4238332408787889, -0.2741885247325473, -0.7829563575009196, -0.3201244186681932, -1.139188219318722, -0.7180475154713181, -0.8596066239123691, -0.49921079006899505, -0.2600902260664378, -1.168172574721451, -1.142182272173389, -1.4629477196637408, 1.154490773551831, -0.9963790484386504, -0.5603084677186521, 0.12012210720307913, -1.07179880195836, -0.6241705770578092, -1.2549185971961208, 0.3102325766963015, -1.0740077546932694, 0.13743401382407816, -1.0570796043928252, 0.8460359391140201, -0.9737036849879968, -1.564423806486876, -0.720186771061947, 0.3397624124250082, -1.3567636486851398],
[-0.01200800784529336, -0.2152232144429248, -0.33414693708454923, -0.2523085007151755, 0.06418098730443508, -0.03128444417797763, 0.5368910816260305, -0.2352092237829818, -0.025405113894299444, -0.2139585259946935, -0.7305095821593073, 0.41359518109282817, -0.1652934217147233, 0.16078840842760356, -0.3041470299554946, -0.0873266722647505, -0.2039146756266145, -0.5349026688769425, 0.20521013543763506, -0.5480889602543125, -0.40861826492761993, -0.31740776655977093, -0.32302166087664863, -0.5760931099998057, -0.48452130626803386, 0.11673287935213493, -0.4271715163509854, -0.645537268093641, -0.3401703951725322, -0.5467608782032664, 0.04108625119693398, 0.031655498918188654, -0.06140860706581588, -0.7104426902242014, -0.5795334918329768, -0.9478887415853329, -0.8441935812861558, -0.7161744552121977, -1.649201112363076, -0.5130825474450484, -0.9969601741198341, -1.7660444797315584],
[0.009461861524939458, 0.18356910102253934, 0.04127547127030159, -0.09272169838375885, -0.08805991552564096, -0.14017465014508237, 0.1242758007767591, 0.0005769739930986231, 0.1402305161274172, 0.0033937127555618853, -0.113612604105515, -0.14208059285135338, -0.17053462065434324, 0.17852492860128097, 0.06285531756516942, -0.10404747645679148, -0.2729020598746283, -0.2390906213691008, -0.13462551065686276, -0.22114385843098883, -0.3127759033881917, -0.13933915961842738, -0.03580481151773531, -0.43708957840149043, -0.2700168551601558, -0.3536506641050262, -0.5820034835564503, 0.08038951043006334, -0.6499863753582831, -0.5939462801276745, -0.5625080218293523, -0.4262062323441179, -0.05389379829417989, -0.490825040831238, -0.3005617260836047, -1.6666940496420213, -1.1762567920153928, -1.5012703831984064, -1.6610604285655453, -1.0841956600484512, -1.1415272832434822, -1.0810084163160296],
[-0.17539504560631214, -0.16178870837153939, 0.13044121794393096, -0.10759143367139933, 0.1498656523498031, 0.11232116341472659, -0.029198758588243223, -0.10290911914977056, -0.37558551571000254, -0.1526819434327089, -0.03123606768001994, -0.10635804476730616, -0.06673909537890997, 0.07390802909749893, -0.07283134020598654, -0.12249015559007456, -0.4196392696340612, -0.4034441559204783, -0.11228154002116406, -0.32948121152906384, -0.059839696774969135, -0.21268892202412276, -0.20024145561402518, -0.3048665066562511, -0.39001922719653026, -0.1034921870761446, -0.4560584833239306, -0.23766727377924293, -0.8289909447416969, -0.2512214312200786, -0.8133688587784966, -0.2833183770174306, -0.5487825513221813, -0.6800037628039787, -0.6946714762056181, -2.4960940167106775, -2.1801571298251727, -2.653511819987946, -2.601002858771288, -2.547000466185602, -2.365576207799584, -2.5633819275074274],
[0.41823087756678107, -0.1976290031033455, 0.03857323249343378, -0.624729071120039, -0.09619312543261321, -0.16043697379160798, -0.09175893723952447, 0.08675726771685162, -0.3267563073641289, 0.023676988966063062, -0.9211463036162745, 0.07492746716071651, -0.16922894632614194, 0.09646984585297451, 0.2858847478539392, -0.3470164079377208, -0.18568679360384896, -0.9429004030622894, 0.5273565642055352, -0.4997035772104906, -0.34271460922280406, -0.19062342741059687, 0.11260908449722282, -0.3621344329862618, -0.7466457732716434, -0.09984675729791276, -0.7135584453312073, -0.5901051437287298, -0.26677568607691293, -0.8929474108979655, -0.7476990502770704, 0.15967992517889973, -0.3800058696886834, -0.8782421879408384, -0.19644579983122146, -1.4367393482540307, -0.11667797453644409, -1.3015744938418279, -2.508053109327832, -1.0380771076184796, -1.0471263553712138, -0.6717511557733956],
[-0.18076976203296893, 0.04571464750404421, 0.11889864698304721, -0.011583716115670812, -0.018411443848930745, 0.04847806648009771, 0.006476788855079667, 0.041475738446536674, -0.18159635257696194, -0.10024308132082498, -0.05432632099143348, -0.04171526329294319, 0.07287627718834788, -0.17013415083391326, -0.2649100308540207, -0.14969719921664495, -0.09138247954850952, -0.5178485840049654, -0.2652721152501841, -0.24102729194201902, -0.11599699408064014, -0.2394252246864263, -0.2680764831652725, -0.16895206352318226, -0.49657586319602015, -0.26265221641797337, -0.22471213017487185, -0.08373596136471156, -0.6193493121624505, -0.549729243251472, -0.6068630714457092, -0.10667849563357513, -0.35637871622848744, -0.6079256802647489, -0.4140336817581139, -1.5096725963593176, -1.22197604964741, -0.9889382343560917, -2.226148867425783, -1.393936045177124, -1.3546019509483918, -1.655874929256116],
[0.44279036873603234, -0.17597539073836074, 0.5680852588452937, 0.16271710367583686, -0.06751913188535451, -0.2230874912084644, 0.012926837693717367, 0.328190343012126, -0.20010583790551964, -0.11485343279772164, -0.15953319050334352, -0.33076723987860207, -0.27977128126635914, -0.03808940078082525, 0.04621709738242138, -0.28076836005315453, 0.27114895486274626, -0.08337505017142997, -0.34374517024085693, -0.3065472334059543, -0.3018173803511416, -0.7269642031061924, -0.35466624145779935, 0.008065225768284454, 0.04578782052559293, -0.7540965448427018, -0.626937283312445, 0.27409450386068035, -0.9450623262466081, -0.31003369804908887, -0.7004262210733851, -0.6685435723839191, 0.13450845812920262, -0.2948644494957903, -0.4526082953688001, -2.608224787405519, -1.6282182475155245, -0.10597211373904243, -1.4040175443868823, -0.8901901909994373, -1.0846182025326903, -0.13968219990330544],
[-0.0767858615223968, -0.40655710730764794, 0.3410698766330451, -0.0915388173533823, 0.13661359965343695, 0.1207696099647256, 0.09846470064379649, -0.07812908088452758, -0.03471332524749899, 0.19856716343539152, -0.5109938876776542, 0.31956668436710817, -0.12315417340368323, -0.2064412127292508, -0.010163566162692829, 0.1181563192917537, -0.376734406521215, -0.35219908133398165, 0.27509444393893934, -0.6861749375609221, -0.3266281003566603, -0.2705831201379347, 0.05158728608226709, -0.09577995721352892, -0.5302512700407968, -0.19585386043251127, -0.7868071290509654, -0.4458760145566235, -1.1358615658718691, 0.3755708645788356, -0.660141412192327, -0.5607151687934722, -0.4212885865615454, -0.08753320228273896, -0.43644699347853494, -1.7128625099554062, -0.4819388322757224, -1.866009345180946, -1.9699218621165941, -1.4400155631394596, -1.6774948969086108, -2.0604120277949343],
[0.21507526735655258, -1.184854812595136, 0.042952845895540374, 0.28186519841582336, 0.1900081047974185, 0.4245770725033591, 0.3434012034267897, 0.08921573160722941, -0.723609581843548, -0.05270626279464075, -0.18935568650495185, 0.18613516433020033, 0.1431729245914766, 0.5820314033568934, 0.18827949118719028, -0.5183879386736048, -0.06412581162548395, -0.09044201984733873, -0.1739256577512984, -0.2731811115764681, -0.17183021722031408, -0.2079306887372419, -1.0319586253918036, 0.21740000580857627, 0.1276737383832922, -0.474477589474134, -0.23052366533853616, -0.33953453704885017, -0.6483119586666338, 0.30921036830119075, -0.15461534364494803, -0.6012793284206717, -0.5882633155509644, -0.7646081587928043, -0.7788858588075118, -2.1533296164595845, -1.718344651574607, -0.5317919203467151, -1.9108547625487693, -2.224334437383838, -2.134681083038513, -2.3328050674962344],
[-0.20745290106243056, -0.08962047897106007, -0.3280066030142327, -0.40503712056391517, 0.029378170295421786, 0.12326977430477612, 0.12137756986971493, -0.023953485629098055, -0.0012538037662745466, -0.25564148074213844, -0.3959052301046187, -0.14876262154831507, -0.07983180539491273, 0.052555742672019824, -0.04884061834758368, -0.16574108683581643, -0.2755908175709568, -0.3514161814580704, 0.02447043649814511, -0.1116901536199443, -0.1443681947744168, -0.1644344381482862, -0.1317747558428372, -0.14112199891416824, -0.4041884375562889, 0.0023918273350149147, -0.17633676602951234, -0.2566880189047685, -0.604091760374351, -0.6845407586071158, -0.3982208338432638, -0.02278993773091587, -0.5362728979697412, -0.6776653964489542, -0.6838985636179749, -1.9123281826416108, -1.718673738948892, -1.8368978395396904, -2.416681796052729, -2.0403819804347676, -1.7499369587648619, -2.390558586408225],
[-0.5619300469278289, 0.12975986215289564, -0.13246840453698283, 0.10650479141460692, 0.6768408310669822, 0.2746186150918699, -0.45629235289003783, -0.5443544529894259, 0.09372966877961607, -0.14643188655903722, 0.24788503125097422, 0.21127368671520236, 0.0993324813131145, -0.11334863822917021, -0.3886229209054493, 0.010089321696454333, -0.16787192376039584, 0.13010560954783112, 0.16231278023223744, 0.10732037219336661, -0.11009377722765239, -0.09396217766236502, -0.05870399945687266, -0.15104524846475845, -0.0679000443343152, -0.20125654546039173, -0.25814345717004283, -0.11266840981919948, -0.7511692542089135, -0.247523505260417, -0.5022481192637396, -0.6426599183380247, -0.6210538901661289, -0.4665103973524153, -0.8419074740114588, -2.253829871847512, -2.230035260738862, -2.8260536466223836, -2.6714351593173005, -3.0540704016498497, -2.621456471055456, -2.812407061214495],
[0.010418280023001563, -0.17605746509332895, 0.05614036316002523, 0.3240766275914119, -0.12411022733588786, 0.0684389309788986, -0.0989231491500776, -0.04771428793398443, -0.08449184034392732, -0.38906012784228994, 0.42951932314904545, -0.17312653550340368, -0.17756071539125556, -0.09505764935951906, -0.2157308953087896, -0.19348025753973722, -0.42301036883301296, 0.26919927633565427, -0.5967747816708746, -0.1316292793181608, -0.23863325023078177, -0.40514416558616173, -0.4098909252637111, -0.670690471553408, -0.14314349732573273, -0.5718026106270719, -0.2493134361398181, -0.2730051955799191, -0.748386208901656, -0.41281765387900576, -0.1523804625099272, -0.5685726982878078, -0.8067366164616047, -0.35084109907443034, -0.6787976199580066, -1.5146794970607038, -1.3895680315475252, -1.6434791497710777, -1.277521103997095, -1.9664890868885132, -1.8288824386537088, -1.8826204147615664],
[-0.19670840361958417, 1.0868996340995754, 0.21142273759245306, 4.343891685746127, -0.8689972441908663, -0.8538419172397238, -1.451632239382606, 0.06516707054490427, -0.1186683915610422, 0.2649378475097755, 1.353378158968569, -0.6592513057899573, -0.48475677973764647, -1.3124584286721417, -0.6480972887746862, -0.4034703142308669, -0.2286969536586446, 0.298363543835193, -0.8415815591637658, -0.49718894581391404, -0.4249508211992324, 0.11095068182041339, 0.28429255169463713, -0.3854944940843711, -0.4969627262579908, -0.3381837705676854, -0.2409055962703795, 0.7659389931829533, -0.5572533669696885, -0.7799405442274757, -0.6690792142413811, -1.1705477101978106, -0.6105231409820651, 1.1908608951482678, -0.45130419159495183, -2.4628344581412493, -2.2882655070845597, -2.6659704872514194, -3.0984531999129525, -2.554395959043895, -2.0468788932458173, -2.5639726802234453],
[-0.04640096686644378, 0.15124107283503826, -0.141517653663983, -0.1278680257366609, 0.01383971462653602, -0.14497904769026365, -0.15371364951718594, 0.053607181942649317, 0.0674640978621816, 0.032182033374134174, 0.019352686523249424, 0.0046195038978430565, -0.07082653812425724, -0.023836589639319884, -0.2322374441754832, -0.2398290785365537, -0.10132970796104869, -0.3085385848586357, -0.031676797356563854, -0.007281948487580794, -0.08725071825886135, -0.26155076046707126, 0.17999354506683587, -0.2641329188136393, -0.3315973073987199, -0.3611232327676246, -0.2195098009224895, -0.050522535500135864, -0.676000030138218, 0.11040582689629559, -0.6275945405917348, -0.4708120327042389, -0.3799609865472721, -0.33809720076497274, -0.39719752193739033, -0.8105611409544953, -0.5555771378991489, -1.1492249646572832, -1.726612068464936, -0.8478579880272203, -1.0861167888917487, -0.7832038257977287],
[-0.020100521203569154, 0.06522298830029163, 0.026082819678546445, -0.16641780451586385, -0.07704325240802598, 0.09181611499673484, 0.09508531258131389, -0.18238087053408453, -0.033302660821858995, -0.2805640968430065, -0.12766500099844416, 0.03946782546709019, 0.05985551981209083, -0.07036716926180253, -0.131657997263723, -0.4057144147137511, -0.08338444655014013, -0.45280049441288933, -0.20864077670257428, -0.4225076545379176, -0.023785621680005843, -0.22203666334970293, 0.02419228387343183, -0.2894859075828592, -0.4666391927321483, -0.014657285786599165, -0.08791241817764342, -0.11551900117163642, -0.5971615603984471, -0.43477139264071746, -0.5483684927194761, -0.33802336271907185, -0.5153594010107179, -0.5051301840851746, -0.5591427981196968, -1.586203885596358, -1.264200189062176, -1.4426266946444528, -2.323732171203307, -1.4482031025695639, -1.3135587495911414, -1.8391738748964306],
[-0.06619223595797259, -0.11864601981064955, -0.001982977174183431, -0.037726253452489596, -0.10277669758081245, 0.11863148285993534, -0.05428321473953142, -0.058577252493136965, 0.03694265116901208, -0.10786584805809711, -0.32723373282054247, -0.17712014375188057, -0.19337817934884186, 0.0765542265857547, -0.313467723219666, -0.23517892451003067, -0.38159520101203176, -0.3565624780807783, -0.45748746287878067, -0.19990575499079208, -0.452392325449112, -0.047936919702304195, -0.2482441112643618, -0.06959546135055117, -0.4038511535060806, -0.8939358086715569, -0.2879283730073034, 0.2057439279105345, -0.5264761052291467, -0.6199068295215067, -0.608283081840075, -0.07592893684939654, -0.14803430731208345, -0.7561695445551672, -0.44895510781149606, -1.558695773941752, -1.1736554832857842, -0.678904837967798, -1.3836319867692486, -0.9046556890714819, -0.9188368689793018, -0.5730254363565507],
[-0.12140502902707767, -0.6216725287238675, -0.05175541847491755, -0.5971621772478776, 0.7003094975235985, 0.09060235094402622, -0.17535596579546073, 0.35408570712483656, -0.5252867760388604, -0.7829281429397874, -1.8489968166353787, 1.538964400464322, -0.03185721278783315, -0.453535096282889, 0.3713974992141875, -1.1123002453786077, 0.006619161333986665, -1.917809775909891, 2.6826727284391727, -0.8483520789634069, 0.96441768162547, -0.6341755693369714, -1.0365729724241253, 0.4296893209186724, 0.5769057361218617, 1.4646555940708774, -0.31089169660617033, 0.08811583833714377, -1.3075407854977041, 1.0010613447264993, 0.3687108519506648, -1.211923259168012, 1.7358304683814734, 0.9010505379400805, 0.8223812512111348, -0.4707606541307677, 0.6617069904577741, -1.4464249092155774, -2.5129324623805425, 0.23794300308774588, -1.8462769140248807, -2.035474430187024],
[-0.7251824177092564, -0.08264786563546234, 0.4974297739247069, 0.9267263694972364, 1.1072957480904462, 0.6313174237212524, 0.24410006955765665, -0.5344633728796602, 0.004968468369936838, 0.2142822624078879, 0.6715452981354044, 0.6180074962402838, 0.28400371796557583, 0.2750539504662562, -0.15999372233052406, -0.03999944907149009, 0.8123779915827486, 0.3538338009202325, 0.8110874165523376, 0.16481309522071394, 0.2473471744968725, 0.08051135198563165, -0.25012396762219763, 0.6260723290770431, -0.06930695138531454, 0.4132513891969025, -0.04320463219178396, -0.09086676616222947, 0.20752575076599605, 0.28367853696598727, 0.8931608336719359, -0.5272971454480407, -0.35378936664098093, -0.5244094386795964, -0.9951734732529356, 0.39781669227187766, -0.5614142811235745, 0.24279483998770385, -1.6777891294708813, -1.772425260645135, -1.4759211274525421, -2.060563212394957],
[-0.17383744032551066, -0.055053288949971566, -0.05009128702438209, 0.019977802775832353, -0.10055346589727848, 0.10825895075100853, 0.012008506950441054, -0.1661720361617693, -0.07684938878866435, -0.17984654249012716, -0.21005052152144932, -0.2783158791130075, -0.0843503836386228, 0.069465434673608, 0.4238301841419186, 0.06416124594909428, -0.07530323627544501, 0.10959662019652454, -0.07846900730261813, -0.17072189428454232, -0.18749579290711058, -0.13317227716153457, 0.36984689668209125, -0.05498444038423153, -0.2821169767824223, -0.3314999938717252, -0.29247920253982224, -0.2955997295650394, -0.3701123510485503, 0.3378953359361987, -0.27717076437544363, -0.7827166213055469, -0.0760658123592118, -0.32907942494978987, -0.357885516195024, -0.03870593537857621, -0.24135173070075988, -0.6406481001512081, -0.9229626525071527, -1.1164834340625382, -0.9670324822501434, -1.1771598859383798],
[-0.0777289747156051, -0.13274639877491354, -0.07024780415765945, -0.620796946050583, -0.4423838688798961, -0.2106513555091007, 0.29036119585428216, -0.39202465199978154, -0.04715126378331681, -0.3238291077441013, -0.7415241184517588, -0.46275292146515185, -0.5224178776744196, 0.12132059563900961, -0.09786275242382114, -0.6196243993743081, 0.06152563561796425, -0.689593228806721, 0.21795357527419523, -0.4889128931133545, -0.14158258803478435, -0.4188255071050422, -0.6038363213630861, -0.37532232530260073, -0.7894738410032791, -0.6428798261665226, -0.7524950687685396, -0.5339626351373231, -0.8808149994838911, -0.6402640432456398, -0.44747972316436185, -0.4445993341117049, -0.7616761260448213, -0.5163143713231053, -0.8752033376096328, -3.999521503476324, -3.487350143214966, -3.9364990499645693, -3.999826148526718, -3.789069734831761, -3.6973383782583067, -3.916047028983457],
[0.2875417726157399, -1.3072758222595273, 0.2987817651322109, -0.08499789505956182, -0.2345555377648248, 0.6072458709373819, 0.9381384035021049, -0.20442465220592576, -1.2065934155754292, -0.28973745604121653, -0.8085865889402533, 0.27606844334324737, 0.6028347065137722, 1.077802970876772, -0.01295270185599949, -0.9374613542191089, 0.18699452649026846, -0.7064285126751421, -0.07937865524162209, -0.017411899317556392, 0.13253732456253825, 0.45395866191658957, -1.4124186945390753, 0.47869815517179426, -0.7407696326020019, -0.4652144615982841, -0.271068716121784, -0.15750472615738065, -0.5472902938113077, -0.1650238126949481, -0.5090188809715703, 0.1314493770686757, -0.4867552589965867, -1.110348987249625, -1.1173545476677844, -2.550446588247728, -2.097246266013761, -2.4531339509952192, -2.9034670067437336, -2.6436649026667776, -2.405222394601633, -2.7331701499255283]]

who = [[-0.24512367230927865, -0.07966944337038592, -0.23448811787575807, -0.5557971336749705, 0.10951043687769194, -0.1350123472305447, -0.905882848854693, -0.10048931621556573, -0.1326932790588203, -0.17786384552195506, -0.11185865429694865, 0.464931830640499, -0.6163228855277697, -0.0915708261766171, -0.38445304419389426, 0.067538633151492, 0.2453212875925575, -0.2630692432286298, -0.3032176693695146, -0.47114371822031675, -0.3477147069329119, -0.3197966053679088, -0.0726804776984339, -0.08104139082903701, -1.3240500568853346, -0.225304181406649, 0.10354708495535916, -0.01808510870700693, -0.2580080710886682, 0.6659925153052092, -0.36550745254220224, -0.6063172700605078, -0.6756261091678107, 0.40500176673055216, -0.005852710279153861, -0.17303364477007577, -0.004956998860733249, -0.933609027029027, -0.5258959719437304, 1.327263371478684, 0.48229711325938673, 0.12012544066529526, -0.42342870081589257, -0.23511774085386639, -0.32203357005477673, -0.15834030599944307, 0.08615800773505393, -0.12773539984484272, -0.3769529650495487, -3.5261654395338042, 0.04740255370227664, -0.30824219464754055, -0.31312892658305097, -0.14045892913170602, -0.3060119757073195, -0.19770579259846852, 0.39718162926762224, -0.43039022267583443, -0.10633644922122144, 0.48146798588950024, 0.002818649926721689, -0.33863103447598414, 0.35001073683474526, -0.23521395876616918, -0.13210171399675974, -0.5689591809517149, -0.20397491460634593, -0.10988053547065779, -0.3922458937607701, -0.14614665864475665, 0.08999236929586071, 0.07101727529940943, -0.8201068575434428, -0.07880949547017929, -1.2219648936795333, -0.14674906346272384, -0.3119963173329359, -0.030735006109464946, -0.11482737203954237, -0.08231486677428272, -0.6137062284112894, 0.06729814344319318, -0.31797187099078567, -0.32318079083013035, 0.017136964545678333, -0.20544578107476172, -0.08555856878343336, 0.016430680871672865, 0.10992144463751237, -0.277828325843946, 0.3642301848695656, -0.039983860786489404, -0.12156130398498158, -0.32576829313816147, -0.003635411955554262, -0.10423695637063866, -0.2439902512670911, 0.003650805411053587, 0.5553008786492737, 0.18479898997693353, -0.21393614787129928, -0.04226967773488103, -1.0787937002173986, -0.018829155725717497, -0.06244360030958229, -0.32798234793770403, 0.14346481176125453, -0.13243470195504792, -0.08890446954856597, -0.4163901994385433, 0.13101629022112063, -0.3141815281720026, 0.025594283439771017, -0.20201420569905507, 0.17329731097936424, -0.1187019977203683, -0.1768501521989029, -0.4590477965940938, 0.02500816584013363, -0.43251483041823574, 0.059700798206239206, 0.1266085722881625, -0.10660282090777848, -0.24147125735684635, -0.08850311443115096, -0.11413029247459372, -0.24476749288008495, -0.056245922355859464, -0.23487716733406175, 0.14558498212171575, -0.036194908155144395, -0.27968093621033785, 0.5454320646028031, 0.09462358708103268, -0.4709589347495047, -0.050839856485318254, -0.03481568009420873, -0.20586232236973973, 0.0842628374815117, -0.11107347097147412, 0.04689290314310198, -0.4154876215836073, 0.6182413042352009, -0.3055057898433698, -0.09343947432052692, -0.147210987739462, -0.5366606647819165, -0.27131728778922704, -0.03950733911351826, -0.1713737885119948, -0.09482454170982219, -0.41757944196708136, -0.11294101944375924, -0.22941872187182677, -0.6354168974744232, 0.10127589129379701, -0.32704400795437666, -0.4120913432163522, -0.12449765736533591, -0.03318142464886916, -0.18535310218276718, -0.04451967617937877, 0.15895568379334019, -0.23285254181279894, -0.15612516881004676, 0.3928689443031507, -0.060030624037013196, 0.05109378088243998, -0.02966472623588054, -0.20563247276934096, -0.18496056192235266, -0.29859158587471385, -0.22398315760773901, -0.058450439752268, 1.1664038185094734, -0.39829291569964703, -0.08917185189857921, -0.06209491228831658, 0.051695368192956614, -0.008614635842866994, -0.06243698748926016, -0.23168964899258862, -0.3988093966285149, -0.08394386632779086, 0.15611157489971392, -0.09041497257774077, -0.31096581880261687, -0.04264609974341399, -0.10616135442806679, -0.13802590822693803, -0.3993379735705011, -0.20653044305874835, -0.3146661803933942, -0.13476474532791133, 0.051074262335849496, -0.08175894166569304, -0.1721198719866375, -0.06031475417901058, -0.05466832021686647, 0.0619613294467763, -0.3476194030565393, -0.16233993031938762, -0.09550048383839911, -0.07619866531276373, -0.35074542487644644, -0.5064720669697022, -0.16575430296910354, -0.05205420123833256, -0.20795186790647222, -0.27529093815763267, -0.13993033956530163, -0.6822479373701249, -0.16953548882051817, -0.21336997132805935, -0.24036561977201118, 0.19462932098301336, 0.22083230013638275, -0.3709884584983961, 0.3407057325993984, -0.19608083484716152, -0.3088087793151628, 0.04866713768165397, -0.7730612037307101, -0.25682111656428336, -0.1916797954667144, 0.3862142790500004, -0.21600345910637556, 0.024548563808095107, -0.16763847281375632, -0.08173688988860865, 0.07956906904349302, -0.10526683745252606, 0.08619422124965655, -0.19256023664975236, -0.04775061271056899, 0.5914166559823931, -0.2895095778056772, 0.04379042164240059, 0.2655059574132143, -0.09246816934520272, 0.23585717873666312, -0.11340991989607759, -0.2791759362666501, -0.11649167388103648, 1.0724837337612112, -0.12591665044064188, -0.14063542306932308, -0.32890393581506866, -0.39488839172256773, -0.21326161230986934, 0.12992650181477267, -0.352646404713768, -0.4043951937145649, -0.05570444480427941, -0.2824948506500673, 0.3182912237468287, -0.1669380950220784, -0.08119432052547519, -0.476761799949351, -0.19819674500268813, -0.013083538432921633, -0.19317278047283978, -0.12635410753294715, -0.22358287238836033, -0.27835524273526374, 0.07507468976465659, 0.0437904403785948, -0.27655359018483217, -0.36267285210687555, -0.5007650888880198, 0.13995422749368094, 0.07219977550662178, -0.1205461066183852, -0.15695742606529722, -0.17927616031449267, 0.04042096300240307, -0.38549133034764554, -0.1900316763183361, -0.009886122538778137, -0.2400440992397755, -0.20326802541043576, -0.24606964679586502, -1.945479384978496, -0.0474446932441722, -0.5444741927870359, 0.03560956144662667, -0.44839182806302996, -0.28002775300009186, -0.021958849312845346, -0.10500643885982845, -0.3160584474598183, -0.19453040712370634, -0.3688542894301877, -0.20297284180811398, -0.05636444970493377, -0.1059691620347073, -0.2973805549949639, -0.11223934498012933, -0.16667730772568132, -0.11447722676817773, -0.2023914792387908, -0.256885205728323, -0.12676574651898492, -0.027135431338795545, 0.039085890152554155, 0.7049898225770576, -0.19520652512971126, -0.23704522861043953, -0.4775020732413938, -0.11099972221554258, -0.7630093664873194, -0.2773577728967235, 0.06983336422055303, -0.7396407115830969, -0.5255774412017991, -0.24841292195357634, 0.09868504489150486, 0.01769787250348701, -0.20140632300829203, -0.20017575520369926, -0.2752572904105127, -0.2327051576893912, -0.1298072119515168, 0.05235598683471833, -0.06970772915725053, 0.049360770350142115, -0.5129999617962545, -0.3044620847856275, -0.5069095969176699, 0.05730970867420175, -0.21327663701126062, -0.18852227127389806, -0.7602529898832565, -0.07852212521093184, 0.1346343059995005, 0.05113211692439105, 0.0973596789339647, -0.5896166369551049, 0.0016475821583996281, -0.08467260877604862, -0.3141715005464562, -0.023173099319700973, -0.22173725881862918, -0.22817580672775153, -0.12395260218365435, 0.14014269382340963, -0.02384256795328078, -0.17056250746881713, 0.15867442882915778, -0.39237644264677757, -0.26456484495893157, -0.20466360758918434, -0.26847488884850107, -0.16100681356309265, 0.21948599117603904, -0.13189256127092303, -0.14629009022047534, -0.40050514558823785, 0.08256964594870982, -0.15586725718581523, 0.0197338263220212, -0.11277595256668697, 0.030403090678653186, -0.06871803565536785, 0.05237977250111674, -0.09807397265961773, -0.27660504447468726, -0.17388116027767736, -0.257222324831362, 0.15842844427764882, 0.31985148800957675, -0.09040164002220527, -1.6350259367241693, -0.2583786347152801, -0.4020536377709814, 0.20333248076938967, -0.12862923656050645, 0.023549535147281645, 0.051954792963185475, -0.5813137050081891, 0.12216200253061832, -0.18543009100025118, -0.03379773938908837, -0.4138219854213353, 0.06775240252528808, -0.17648119927263686, -0.3108701097697097, -0.0864250615385637, -0.2054701582330044, -0.38027522390386126, -0.5530420516736861, 0.059877298304967214, 0.15115223303966957, -0.21406836985497613, -0.25921523932211765, -0.03795546923078046, -0.01643235010273253, -0.5918123198330676, -0.4012234568924841, -0.169730391226328, -0.44632197834637777, -0.26425178556558915, -0.11979611998934915, -0.026435588061876077, -0.1389886198263043, -0.8984032243387395, 0.09377326110992296, -0.5208015991017001, -1.552075310336028, 0.7448514930069449, -0.10770619775996995, 0.05648840555438097, -0.0810295816799912, -0.008731752349904176, -0.15815936892129157, -0.11398540394617136, -0.16492694620404247, -1.1423867723598067, -0.07765786322735983, -0.08501310909445514, -0.19593866218228764, -0.0597660297143565, -0.324526307423092, -0.3450509411043062, -0.2436548013639718, -0.06777636491467316, -0.0046873929059709305, -0.09141196033743564, 0.1892851841258611, -0.15924051059851257, -0.04278262507295437, 0.07590376391144033, -0.06264059577435485, -0.1267203801028146, -0.2810355159098879, -0.3472136260570686, -0.235319603346662, 0.05806192288363917, 0.04872080147662948, 0.006025619904430336, -0.17351769965404587, -0.24154104069113402, 0.030182990325381173, 0.1969077654986041, 0.6148281589605632, 0.6046222216252879, -0.10534636531241501, -0.31296591668833734, -0.17789007537115725, -0.556345697568148, 0.0379770127673639, -0.25360719376727153, -0.18020112805591743, -0.29467306257013237, -0.2701545263662142, -0.17950136005366948, -0.24785000885990943, -0.44507494199319714, -0.29109077367523145, -0.2397467760763314, -0.15555384030223962, -0.2107333257529437, -0.2529887767990814, 0.08321369069644483, 0.011445151171332543, -0.38247920195432167, -0.1515741008367962, -0.22046765832080537, 0.08050095075728693, -0.07991369462468445, -0.04888655838701553, 0.038963546447855546, 0.041597125918955626, -0.5195736018176776, -0.33206944780114633, -0.1021043930139698, -0.10021437865908728, -0.2087265968679597, -0.05366982388386948, -0.3547164313701932, 0.11381586337986445, -0.21058841301666914, -0.2651666455814097, 0.027837486546938982, -0.005842253628299292, -0.5202973980773746, -0.14014358401018578, -0.15127741327848232, -0.03751815766545031, 0.004742190822426671, -0.34013679102286604, -0.30382957701541224, -0.0670053623547227, -0.18386441763308303, 0.5400227632894973, -0.18238141213710646, -0.22427180074124306, -0.31407335604935754, -0.03935496502718429, -0.18665396601038156, -0.13815811361283237, -0.11753938104686218, -0.20673986112925255, -0.1742139868123808, -0.5704024892376749, -0.19286980476998888, -0.37795085016908564, -0.45874691339128026, -0.20247486897711608, -0.09379587693687459, -0.15033860483061795, -0.5616849016300232, 0.1913364977205226, -0.7295126320566828, -0.12119895632192149, 0.05055067856614675, -1.605955832385078, -0.05887714525226752, -0.10146888343175677, 0.023872444879874007, -0.20644133915413015, -0.14276798168794805, 0.5821619332769614, -0.005169602869094979, -0.15492438570853928, -0.07696929305688967, 0.02505744445812451, -0.9649922908124162, -0.15365544594087216, -0.13429408482831884, -0.5036133403655826, 0.08067967233422832, -0.3576874994644435, -0.32474142394163136, -0.06140014517453923, -0.037263651236419844, -0.10025315864456517, -0.005495277265611305, 0.01945428394564271, 0.037582365502254515, -0.23153792086108546, -0.03850449694503234, -0.027712113877249638, -0.23915805358227912, 0.8898110856532563, -0.30235269687036265, -0.2193837869866516, -0.3507901742938311, -0.22601792896957484, -0.30426087977706745, -0.10748022866380698, -0.2543014872930196, -0.23015717607563752, -0.23783046773334537, 0.0923354105342588, -0.09610906570882248, 0.027963872538151996, 0.014817355409393156, -0.15956287688269624, -0.08297350135202188, -0.14031722637606822, -0.1529301489562268, -0.12492593526963569, -0.005036093703755064, -0.5011946934807986, -0.15237998503753838, -0.39870046256253044, -0.14629070777847422, -0.09240270848552591, -0.2918193291247092, 0.057325900491879654, -0.12692838662779685, -0.17389413255777691, -0.2933617986780919, -0.37201503570523303, -0.04594642356002058, -2.0388888808244046, -0.32162910744418494, -0.309342038056556, -0.8528365504890169, -0.25389200385495264, -0.2363420136938466, -0.3733270041690476, -0.02500984343808695, -0.0063659117980612275, -0.35593064071240405, -0.2627217234519414, -0.05290259932542262, -0.06308567849015376, 0.06936722408650646, -0.10763195861071717, -0.0663745668234767, 0.09486190465173959, -0.018597862959092114, -0.16169713057196392, -0.09724442045513645, 0.3878134421572017, -0.03957701955385415, -0.8695493759466918, -0.38077757503601517],
[1.1987404554973233, 0.17748536638432882, -0.31965881776857435, -0.5532748752095, -0.43183338325855175, -0.10716620900122976, -0.7210007235471416, -0.06532881958794516, 0.05348776067143978, 0.007427035348895967, -0.06354689644819732, 0.6084477964053522, -0.82605283750689, -0.2496068459173907, -0.004534955973025128, 0.013164994597169181, 0.13224461332982432, 0.15292988572504046, 0.29892725352043487, 0.1366582491276202, -0.27090935744548533, -0.04984971786763393, 0.13615701935000618, 0.19336797570474226, -1.212431483012769, -0.023787814745549047, 0.1949874099804949, 0.06617119535559163, -0.10978725708117038, -2.2386901635952303, 0.06431530429794895, -0.352307609624854, -0.7581132116351986, -0.3053707548556961, 0.2248264326528119, 0.20548377567759624, -0.253196173537693, -0.9668916821542897, -0.29767477118229124, -0.5044561545989269, 0.16628170138676887, 0.4346732133149017, -0.39017260757847355, 0.06280230478856627, -0.30533266467702225, -0.36800661938029916, 0.06285324230394904, -0.08438654458854683, -0.3610115564544146, 1.0928122166772904, 0.29286213046575504, 0.23693961493583204, -0.22387151877246553, -0.1795396702531484, -0.2559000700072352, -0.21486537086748383, 0.682491276752858, 0.006452489721033402, -0.23594960298401324, 0.2158231174294307, 0.047021462892572384, -0.1440094871872363, 0.010246494254679408, -0.2593774335461852, 0.014138714577493845, -0.2702424698335753, -0.3640870195200231, -0.5694515550539483, 0.3943817739616008, 0.3154799084914474, 0.20736078839768482, -0.06224517852104336, -0.4972298009381863, 0.03709500394681804, -1.2819434021482943, -0.1429436063129822, -0.12231293361806268, 0.41258412307493214, -0.18477582017791805, -0.023311753154258718, -0.8571324026242646, 0.023608955871814774, -0.3922130108538961, -0.7637829885004075, 0.29016460936106603, -0.1840644692268392, -0.3054716871760583, 0.08215685768961378, 0.34757314271712847, -0.1152246495024163, 0.6986988751355181, -0.002258365575867641, 0.026544214748512866, -0.23815704100645219, -0.008412112802046961, 0.17077637999372056, -0.0433562978760703, 0.2051595985424083, -0.5720532794582606, 0.054066709155225226, -0.07392432916843708, -0.12016884722896551, -1.9290703081657827, 0.088024149733532, -0.07083884346777974, -0.19875064600846498, 0.1627571727864528, 0.027705427811854207, -0.04018969520213474, -0.11152433410434248, 0.13891711335809104, -0.470018693001418, 0.08963823679387013, 0.008844750504884032, -0.39259840134075724, 0.05123826714670325, 0.030379045801920242, -0.4174884401897238, 0.2801478585609914, -0.25556351755030415, 0.16299189843048642, -0.40513354299309334, -0.05233576419924645, -0.13180079199100989, 0.027694744597951942, -0.04118314386464487, 0.15437075335138856, 0.05597029613132895, -0.3364828504147892, -0.10762189713447752, -0.03866865300250322, -0.12127521976870428, 0.30117046785964485, 0.08894616076386846, 0.17838544136914178, -0.025775213228987976, -0.00802715966394335, -0.021917719087445577, 0.005222501870044834, 0.1733285625985835, 0.01239135025015741, -0.13589715041516748, -0.4329722649050764, 0.006629843870091404, 0.013930209762709247, -0.14272189577238184, 0.17775339649220948, 0.08249348561017261, 0.013002614858968678, -0.12355847106119804, -0.1319796760781471, -0.4518252488842275, 0.0007465689312025421, 0.06971501547230241, -0.5925425133511879, 0.05980470311604575, -0.09348652737758646, 0.3315194471008319, -0.011540913011043093, 0.1441642095942232, 0.05327111325276314, -0.18441743989030468, -0.523001003488971, -0.017933488376928437, -0.056205362278747166, 0.4858922658334707, -0.10974949008915844, 0.09740912966022715, -0.03932267105146929, -0.06963708572422651, -0.08635672366399266, 0.21451621393810563, 0.8001625270118877, 0.1440672793126692, -0.7863941577876487, -0.15332339712191753, -0.09908260069120749, 0.48090747001882816, 1.1037120332083263, 0.09736306207385341, 0.18509860163364983, 0.14715237008983112, -0.4357443697989199, 0.20956391082527276, 0.24788309215717907, -0.0960950221073088, -0.004805325452633399, 0.187684065571974, -0.0792720676732859, -0.0510620032410728, -0.22893769231300024, -0.3104950184277084, -0.014515946595734712, 0.5515983797381876, -0.05116951653972032, 0.17910195314744504, -0.06471463184618716, -0.16549999208030258, -0.014236293684601045, 0.19237502677609802, -0.4787159153542264, -0.10842867917310797, -0.08093227213476065, -0.09783676663143061, -0.2496427968273211, -0.44268938258048757, -0.22764776387774346, -0.11214981629475378, 0.05064499907934655, -0.24706744815032217, 0.026453218640644083, -0.452499886378602, 0.06343751992731936, -0.03388533530888504, -0.6121434257142608, -0.8025803659832482, 0.14871692485880897, -0.6554527012210498, -0.7327441038474117, -0.13846957594686862, 0.18119276067686596, 0.15292692771584682, -0.6654280545601774, -0.08253268599962721, -0.24752723918302713, 0.41649947222478406, -0.43883686839171565, -0.002882612238081896, 0.03800019708329766, -0.369432406196853, 0.049367048617962, -0.08136021478073027, -0.09792756737746182, 0.23570430606932163, 0.13776166210819746, -0.07121070454796581, -0.06858380959782097, -0.014030094690668184, 0.2963080584761396, -0.14872664892526988, 0.2633239913397135, -0.06399392110916316, -0.30634778358802306, -0.052461354983858205, -0.09788525294966684, 0.08707100165274391, 0.3615070781631229, -0.19539876778439674, -0.11166434079300808, -0.02413424056692549, -0.2809529801081242, 0.01937915622719657, -0.141643226677846, 0.024868946337441127, 0.2540452311871977, 0.19504976144336025, 0.1816855102829781, 0.054211528436119226, -0.0725568706461297, -0.08270618960943518, 0.2815290788438053, -0.2570355288664277, -0.0910940876982784, -0.16161018587373607, 0.21062988966894042, 0.19195715123795998, 0.2433019965278489, -0.01761475862549438, 0.8853834238831058, -0.082540590957947, 0.06720353388028329, 0.1570019580605831, -0.15434342196540415, 0.16248607271810345, 0.07136875465635219, 0.17195829027227413, 0.20531081080599423, -0.09827827993289076, 0.15418996175009186, 0.3188315342240348, -0.19685218097445817, -0.28877256594342515, -1.371055492700186, -0.009182039004476145, -0.5974135180232214, -0.1293107006057434, -0.3454634437111103, -0.31195888546405076, 0.2297422939000962, -0.1505422727197206, -0.252246875463299, 0.5394083469011388, -0.35067210113410613, -0.3389618055057426, 0.048288715545903776, 0.024419105636954364, -0.01959152065561884, 0.13309598613387155, -0.09250427120699324, -0.07792638002266611, 0.08072082090829556, 0.1321476918897336, 0.005027612241546091, 0.0620321674184421, 0.1033458706050113, 0.5134854708699578, 0.020156980849834118, -0.045410181079890186, -0.22670983180427073, 0.27256912158122804, -1.1239436983746496, -0.025464836599985274, 0.00672790503804872, 0.09247793775641162, -0.2824597384705835, -0.11556633453912821, -0.008260503617841978, 0.014812725013104438, -0.4324855259980901, 0.02953023919995693, 0.02191734285123322, -0.23240869442066042, 0.09214064267892313, 0.041170211730584924, -0.0343984034268271, 0.2875259343762243, -0.6486483176861055, -0.23733437197967047, -0.3703889890303762, 0.291939407490527, -0.07410470182458853, 0.05571366207302913, -0.5109881781049594, 0.07097701764068105, 0.03146311116572066, -0.01937580087789918, -0.4755565866454481, 0.04842245070487957, 0.07383487397713319, -0.12979791601620352, -0.011800279352149159, 0.19520978769792643, 0.009039817568473054, -0.315935555844874, -0.05846712667031176, 1.0149521692163253, -1.4120898110959728, -0.46239562937060735, 0.4819001808453527, 0.35059968243981876, -0.4012429722090057, 0.13270353715614241, -0.08699757379686511, -0.2836025289932873, 0.25927569426821523, 0.051273193135510306, -0.0589616409836777, -0.1879605893223504, -0.01719696895347656, -0.042207427906222054, 0.14775867881711724, 0.8279277093666221, 0.033772689347324206, 0.0038257099785378905, 0.12959234905471867, 0.2551774857648193, -0.9474653799070434, -0.03901301070079282, 0.07583843227642677, -0.2561871451540717, 0.4047970765178117, 0.10448746549358572, -1.8941420602001242, 0.3396270884957846, -0.012528896951842316, 0.0861984245994673, 0.01671322012850404, 0.14298353579965456, 0.12315596186820982, -0.10277948747844991, -0.14462615944900972, 0.12976605950593945, 0.08853750220426558, -0.44099786590452883, 0.17273497654041844, -0.020655869146000334, 0.0308060279007813, -0.2679553906961594, -0.00409415542953326, -1.5080181968400035, -0.46799188790483237, -0.07633554746603799, 0.29386283237397065, 0.17640956616011566, 0.6561986095667957, 0.20527912364326914, -0.15668513905325632, 0.2542945873221718, 0.005976077659217333, -0.029318089119544064, -0.22572263469350712, -0.20016861940355643, -0.21270348821176263, 0.1536286672303053, 0.012254956712712971, -0.7848009624202286, -0.09304606375080211, -0.3097710257734238, -1.327636642097078, -0.3298964471639314, -0.32995549318554207, 0.11762966710225024, 0.0591568650710035, 0.12456156383551317, 0.06667240143116468, 0.39454714109004346, -0.07000076815762145, -0.6563525757838427, 0.08628637535184205, -0.09829102453742111, -0.08217022926263122, 0.3174920340273362, 0.23435116357427846, 0.22020237798473324, -0.19416680963615338, 0.1215870576168924, 0.05990496124893503, -0.09417109545988152, -0.008095410599213775, 0.007191212713040693, 0.354127502799064, -0.005272075873003335, -0.10476340488781788, 0.1612839372993588, 0.14238896590761765, 0.050768887038647605, -0.0749620863036675, 0.9365770546828792, 0.133086314770766, -0.15202209134597405, -0.13107382677424406, -0.08047596608652983, -0.04435426323154942, -0.24342752378285226, -1.435045280750087, 0.6342694802171961, 0.09720228508471981, -0.19834182370688372, -0.30994630583389526, -0.5720790686766828, -0.904002380685781, -0.5086563668330935, -0.20819702309526553, -0.06398493067882714, 0.0877622793166406, -0.07931722909651803, -0.6016987482510437, 0.6877475812864634, -0.09456808192121682, -0.2762682988227247, -0.07357577679705728, -0.06247060045659509, -0.15554463193310036, 0.1502085652631901, 0.033482025439603706, -0.2864929601976295, 0.10370821882807459, 0.009068920723841401, -0.024632496693926115, -0.4424745435304972, 0.09985386618301013, 0.24448531197382184, -0.007678504874067056, -0.24929137147688551, -0.4454724357667673, 0.2682843262285499, 0.05927335690387934, -0.09583301233631171, -0.9855591105942931, -0.2944378449832017, 0.6382402228740217, -0.07984362065695502, 0.007699690590202205, -0.12777483784133253, 0.14565857658437734, -0.8666220504709737, -0.13287189557434614, -0.43352409010830634, -0.040799660243806116, -0.04599977010504256, -0.22421822440720188, -0.17986966400098828, -0.3680708072865119, 0.06204181926684086, 0.6486884723111331, 0.017947701303189314, -0.016309745801983445, -0.007841975889374665, 0.4314891629367514, -0.07269392767142036, 0.011609902106576244, 0.4323602398855082, 0.185899144592467, -0.1303889781546765, -0.4217118230525216, 0.036985216399535434, -0.37236118957189906, -0.004229746172522571, -1.2869367310368895, -0.8452819977883784, 0.0044267819713349485, 0.21352910917024084, -0.24402354996238024, 0.042145457791523594, 0.08537624530609074, 0.22179115135772282, -0.41223500056153833, -0.00011287634316328857, -0.16560516675367293, 0.2422577504039348, 0.13600545980646378, -0.1811756792302609, -0.6206469807066358, 0.03178061140482854, 0.043066257350531406, 0.3107686479260682, 0.24081912326801644, -0.7272290314911191, -0.01913283040230419, 0.08415070612065266, 0.25910216121871826, 0.061703704892364736, 0.17511621967839303, -0.16235079006861905, -0.44568461467067305, 0.12232291672011748, 0.10197024751704822, -0.2700171524642453, 0.10021037199506584, 0.1321971596997753, -0.09770639989027259, 0.12423323122245812, -0.033533463079244416, -0.1173023135776873, -0.4992428499013785, -0.3002827452257906, -0.2484233063830544, -0.34901575481289016, -0.44584802414431024, -0.2048284565398229, 0.1789610044092912, 0.0493512794705323, -0.055633729424385076, -0.2043374480962639, 0.1911973524840301, 0.02077057102107353, 0.05856875750797063, -0.5012389756178903, -0.15719762879084828, -0.15230702017372935, -0.05617338545857705, 0.04337079047556917, -0.11776609413757909, 0.01225537471344834, -0.7139943570006767, 0.19119138993255674, 0.02238983700181339, 0.05940130763207176, 0.09331603458314346, -0.30242583102911286, -0.15664991606054396, 0.004680416456934652, 0.021026420692745344, 0.01067612821662122, -0.7632428265277741, 0.09913763054268804, -1.812902023452413, -0.12629830838110329, 0.04513290017308721, -0.05552576926262827, -0.2813153872209906, -0.16933911211446645, -0.25969012469226266, -0.25194346303658677, 0.12492461458849503, -0.20411034511763815, -0.044563523271248504, 0.6866922669243704, 0.023284819473328593, -0.12106911283374683, 0.07235335815433656, 0.07370324721089962, -0.06275844262924514, 0.016114463329148575, 0.07323142160542515, 0.7597866481755514, -0.005498855828690864, -0.14390608787346346, -0.49001535168987637, 0.15043302239679449],
[-0.762892244294854, -0.4359044728501229, -0.13636524767887945, -0.3935198652380629, 0.047879752507300206, -0.10208176964210264, 0.5061656447887278, -0.023689432641490406, -0.07932707013313058, -0.10964508616252468, -0.12577340755456487, -0.08162026736240507, -0.07878691210281505, -0.3101994697820233, -0.1660765271897881, 0.09313453555422418, 0.10933516698579197, 0.003833020586373044, 0.19674904404775825, -0.1184616707596534, -0.26378285423074366, -0.017146676146176626, -0.05147031250060408, 0.16015744441712454, -0.881233488291744, -0.15231748845022255, 0.009388582161381684, 0.0009308780372748639, -0.013350672059122356, 0.051844805926591686, -0.19710204271040296, -0.7457321591469342, -0.7304818457972256, -0.6864258233690631, 0.05467764118824085, 0.059301855451665386, 0.9987489120277937, -0.6321176321480931, -0.3750399390945145, -0.5109820338835542, 0.7808350528347884, -0.105453255386212, -0.09772795643355156, -0.18990237372960572, -0.2361786958443668, -0.21587806700813666, 0.020907934462390414, -0.04829121220502626, 0.3925365894162843, 0.8843148290568318, 0.1635646874338668, -0.15871568166498695, -0.23429079763151048, 0.07673290472199366, -0.031539997872117316, -0.32721405162714756, -0.5648295280230786, -0.3285202871174683, -0.19581268071536553, -0.4274137775261104, -0.09876752269247684, -0.04252988297347333, -1.9177218036680472, -0.1483919408649738, -0.24272662986519208, 1.419403000312371, -0.48423530920263963, -0.0056265578697747235, 0.13389276673388648, -0.27876004790745323, -0.024589894120907774, -0.07790267283782196, -0.3118795578102576, -0.13197638801351186, -0.8212891687608906, -0.21882706370666813, 0.028859759778111112, -0.04690120222872025, -0.3900758855729253, 0.03634157983064368, -0.33633486076279107, -0.08929001570717374, -0.0909280369754691, 0.5260150951731618, -0.20076997818530345, -0.09703241667809571, 1.139936319388177, -0.033275956149508595, -0.3608423676533554, -0.29244346091574447, 0.3146665649584409, -0.030622629243829403, -0.2521840265786442, -0.8307097094420507, 0.023193200718701252, -0.06951297684184021, 0.118607667360987, -0.054121335775913645, -0.16403830970797129, 0.01399229091534142, -0.18276729934044728, -0.17013269820731486, -0.7403451547244446, -0.14473162285433228, -0.2353432432127094, -0.3142114013463902, -0.1142543214296369, -0.043380944341345366, -0.17172544402118461, -0.058292340843823266, 0.14520784227865768, -0.10645258071055129, -0.011513700011706164, -0.3608291755909224, -0.009612851666061883, 0.09693057370999836, -0.17577199115245362, -0.6618358125328244, 0.14790130221842357, -0.3284769504943438, 0.018379258080702728, -0.24085720522016224, -0.16752590656009783, -0.2509364087708709, -0.28371887792560085, -0.30256052717952153, 0.08071059206210363, 0.14880256262893005, -0.1292653379939582, 0.3274864946756993, -0.1253290243919567, -0.001071227795583736, -0.11498387815984817, -0.01279640438307762, 0.14120245119856878, -0.11251667932461323, -0.0024387207540694403, -0.1128355664644315, 0.09987392164394272, 0.02408390477822417, -0.07730735644922851, -0.14695852804252554, -0.2107956429923367, 0.14966812907888938, 0.1653310801600113, -0.32807827373853343, 0.7358597313600485, 0.12666332964282798, -0.04665415514446229, -0.13488375516906337, 0.028358698767900894, -0.35152867526991255, -0.15896105049505252, -0.13955513703173975, -0.4100174074466098, 0.22563931938886203, -0.07272047029940665, -0.18823014595852477, -0.22340182785530432, 0.07980787650167533, 0.06818818334634988, -0.36408638268973853, -1.0701967086273967, -0.36939456235609425, -0.20573579140663328, 0.34993682972751905, -0.11181686731238531, -0.0021955675745197263, -0.11421393320736914, 0.04348928146008927, -0.08555224778566466, -0.056287358823349946, -1.3148123678482828, -0.03847963917269233, -0.7595217733222356, -0.163642515349902, 0.018081625101298578, -0.11620116607333655, -0.8519819404736004, -0.06506025771815549, -0.04033519511499189, -0.0297321216373236, -0.9435119270508168, -0.08587116297617452, 0.0570252096817489, -0.08708672665960907, -0.21696498045871657, 0.018303960646220092, 0.038926880194280804, -0.11284763347933816, -0.47986977334472825, -0.229907036345656, -0.10961259103523312, -0.23001807542588268, -0.028452832106426445, -0.16951711553701687, 0.013718735521779964, 0.08499718087415456, -0.11133188456379244, -0.04308004337312454, -0.34329940442203494, -0.3355177287250735, -0.13616133425486115, -0.11162511015454947, -0.20159573509741172, -0.2435976163650986, -0.15568674993983542, 0.16048580385831906, -0.28654214782594234, -0.18010962867700533, -0.12983242301861125, -0.4235225912462689, -0.5488440008153745, -0.1106725145850897, 0.02173714957192161, -0.48190171083680144, -0.0700524169428008, -0.06813781824952966, 2.247744356599836, -0.19077697823803275, 0.39670630296594245, -0.0728457174223028, -0.15702499218014096, -0.2876536821392495, -0.07585984952789683, 0.9933438112983323, -0.08258253341433584, -0.10680847652637719, -0.007800500496903772, -0.3700644714162623, -0.05853196266219234, -0.056429370800598865, -0.17376059251642162, -5.818842279641344e-05, -0.0016042656838437222, -1.0737048543501206, -0.35559583811335466, -0.16218049082853905, 0.4147780355570797, -0.04990373675050211, -0.11706570213143673, -0.6472783982664914, -0.1664496016195238, 0.04482473746860533, -0.5299961202840943, -0.14345148687028964, -0.036580942801911265, -0.310098913209857, -0.5119927437504908, -0.2855235338226821, -1.290328636498895, -0.2564779056067049, -0.16301307086691727, 0.155096100698745, 0.06777933421840118, 0.002042987321833269, 0.05487831650658857, -0.012669364521166234, -0.2009908361675896, -0.221844842415472, -1.0157539198317365, -0.4739499520781252, -0.15918880733525512, -0.07967636627648637, -0.10895623666177903, 0.12737045942207448, 0.5126428039925741, -0.0726912871721772, -0.0411764830395658, -0.2502618905333859, -0.07651691965162108, 0.0406397189009075, -0.28162809732423294, 0.002602381459930553, 0.10258821671120076, 0.13948389797387575, -0.5315500411203192, -0.1998910970190983, -0.10022445565523752, -0.06657705514316518, 0.21740420952384842, -0.18091062732742508, -0.8160908314623625, 0.06912084219206557, -0.4355368619448873, -0.02458397804349127, -0.13724684500448853, 0.12936130236637836, -0.14071109494512543, -0.2048181051465463, -0.07952708168332857, -0.43129898102795344, -0.3557812762910175, -0.1209285820853958, -0.021234430670881523, 0.06340493666260219, -0.16995966215284675, -0.11179238264284928, -0.21373592162468555, -0.08014937435769712, -0.4255190094917578, -0.2453051604180788, 0.0014116961227194126, 0.0515159015420625, 0.024735914586919445, -0.8307885547699302, -0.07186994910722107, -0.20866438619695843, -0.15720763207675792, -0.005265698701397553, -0.8114710475823514, -0.08654823114316701, -0.031171961876499617, -0.4043091035040212, -0.4906480274142235, -0.15467053427851468, -0.0697951415058936, 0.04745309754509691, -0.33345118835950627, -0.5718446557297344, -0.3521219545608262, -0.2692058921987762, -0.050941416874156556, -0.054300774685059686, -0.06033004166791627, -0.08236435284585922, -0.30115918250774487, -0.08566954405279004, -0.570479022619933, -0.4795650214152386, 0.04343696798959027, -0.25166761138305177, -0.7257653350406967, 0.08700245032529726, -0.07165265911351894, -0.0645537015745488, -0.3239730169713911, 0.05429048098453484, -0.10989314433294892, -0.2117963596676224, 0.4641702839554991, -0.14335969172122132, -0.5319297197512268, -0.11394979082876629, -0.1351533116346735, -0.7913238042244042, -0.17679700279482188, -0.04449661499214176, -0.10331723293934215, -0.3409089482190741, -0.08486548220851055, -0.06860628964301652, -0.20270855898628273, -0.13390323141798272, 0.022779011947372738, 0.02163649657219888, -0.05883029122086695, -0.19266138832455718, -0.04714483095635262, 0.17209864648138787, -0.023380049187771125, -0.3960742531250984, 0.05983474926728843, 0.17027470055074664, 0.02510142166852286, -0.1291031263475403, -0.41864464420959546, 0.6844644530892634, 0.2882289194251409, -0.05787454173786236, 0.6394333442160712, -0.004445786992371463, -1.3319154078621536, 0.1748048566658364, 0.10967398507034606, 0.15664805247072655, 0.01010080252480525, -1.1577858144172748, 0.044026540276707236, -0.20049809364846188, -0.01626857082671135, -0.1729051673834237, 0.10024878458706787, 0.4419539935414751, 0.06975441559231285, -0.2352002517005162, -0.22520051088944065, -0.04674119237131164, 0.046269534603763054, 0.9122757523365227, -1.103587133793321, 0.024424246920863094, 0.06322743084494264, -0.11005917986238853, -1.0513750409022533, -0.12762378040024527, 0.12056523606366301, 0.11449363078637574, 0.061375498607971576, -0.12547791466472427, -0.5511476136033018, -0.10830469602412883, -0.1299866069796117, -0.174285458942903, -0.2465859546486262, -0.28928049484826646, -0.04798850807269864, -0.24028691938410907, -0.7421237048835401, -0.5190730430993982, -0.18956300537480522, 0.05306702426570461, -0.06555533857672896, 0.0603399863150333, -0.06402134198722174, 0.04128797475925934, -0.2700280785410247, -0.604091014714786, -0.14731321340330575, -0.0648820140346666, 0.03678535663515583, -0.03370782018841497, -0.024954737008542265, 0.14813799484607704, -0.30349239794873495, -0.19647025697327355, -0.24332731192069976, -0.09532105195589323, 0.03538607198162944, -0.03457533492291325, 0.0797976392801768, -0.06735178407067088, -0.10326430046464613, -0.24321842386459613, -0.15119326442224523, 0.20469845424031904, -0.45984464463029473, -0.07745337463076951, 0.015554225097849731, -0.31935615067268425, -0.06779793207583062, -0.07507916338301823, 0.15049198762033708, -0.1568653232343193, -0.011665760197846336, -1.9136768438407912, -0.19329699711382875, -0.08696339356212028, -0.11564141972222249, 0.057001346018810045, 0.06596499897458466, -0.053930401607027195, -0.2946903261914875, -0.09918289098667915, -0.12470777742754077, -0.1448489251833194, -0.9962235707961717, -0.6480092591299274, -0.060771428123643785, -0.32625413492212263, -0.3990318208013952, -0.15552783076129298, -0.02545423048956408, -0.06468357376983183, -0.0900044941891318, -0.28760503516208585, 0.03420052862304049, -0.14896821805557492, -0.09568036352549876, -0.7235463994088985, 0.017730990975263455, 0.16842075060326284, 0.01323235311959438, -0.427475906847235, -0.36949492501812503, 0.009739231150753548, -0.11982388457336844, -0.13658504653025358, 0.2640141163944617, -0.1055153709757188, -0.6183504009698698, -0.1374444369196469, -0.17495325991379998, -0.22942452516288933, -0.05971563048306535, -0.9291106456345841, -0.08111358744403424, 0.07741499778094786, -0.14735071274627762, 0.10828700186073197, -0.42357798999060414, -0.012118298156456313, -0.04655844270134045, 0.08222082029245296, 0.0904283759478167, -0.011779068835924437, 0.23839619164602288, -0.2935073973003332, -0.028567922053450948, -0.1612507505518024, -0.051173413694915446, -0.07226990715573842, -0.2663791705085491, 0.03890381305352302, -0.4590078169740921, 0.13515095582831857, 0.20566401934034909, -0.02818138962647016, 0.7620445679040919, -0.18648952263282823, -0.06881180775278116, 0.06151618715710699, 0.010342269326736401, -0.5751145051274897, 0.1201559974118769, -0.10307882377238206, 0.9650334880462201, 0.047520101382769624, -0.22008910740769655, -0.2712935705320429, -0.15713913081357003, -0.5904033938514467, -0.7571722206953464, -0.08422586182338687, 0.01546123939717975, -0.4123968482973377, 0.028440309288442777, -0.3298054972473365, 0.0605278878130558, -0.09531037981024804, -0.04182034184978889, -0.030198848244616525, 0.022871288842464917, -0.36772543552913334, -0.044730466979857435, 0.09679014298507614, -0.14364506695723459, 0.02123976547047469, -0.018030404157495638, 0.036323055522090586, -0.5232052338572931, -0.11278292469405975, -0.09045276673836357, -0.18264738687267246, -0.05040890435541429, -0.2653873670228768, -0.44626674572293096, -0.27084458435637254, -0.4734495776219782, -0.14470914758776968, -0.057496299823813136, -0.04080663267027364, 0.228032423386268, -0.06522325050922456, 0.023117943744484896, -0.11060914630396679, 0.05125665948887269, -0.4705622790752564, -0.0411120898064994, -0.16214810911277946, -0.012774094806507116, 0.030660319384802567, -0.17229048953493245, -0.04432183726626225, -0.03770035145718638, -0.05941938995441224, -0.8122196706021555, 0.05201782902398329, -0.12686433085790075, -0.26205753678816124, -0.035817243847404215, -0.09792065284443688, -0.2316317214793318, 0.004125423286395325, -0.23444892971196002, -0.03275033926635271, -0.9480948870432401, -0.21629362161476523, 0.06980306676694076, -0.4577407503782422, -0.016300238405459966, -0.026935486146302208, -0.30122182417336346, -0.1213436532905276, -0.07992999745650958, -0.016630687209882287, -0.4657400483016062, -0.05487653093695661, -0.010997964166988068, -0.09356530504409341, -0.044042362212205574, -0.22735166484585412, -0.04162066438969493, -0.09352807876731027, -0.13301035550718995, 0.09711386023838214, 0.13207096606184762, 0.12149927655442569, -0.759554439289995, -0.397877886157941],
[0.14100232978297783, -0.01541237533087596, 0.11070104787146039, -0.4465291354954417, 0.10557656020456546, -0.08503080599193524, 0.4415990770001594, 0.1465268114429399, 0.12216111141697972, -0.2327784556303243, 0.09143986028945028, -0.1327549782231971, 0.2995981075115833, 0.0622421304564747, 0.3282478294230013, 0.18331137658544838, -0.9916767683917835, 0.05732364408019079, 0.1880979843205964, 0.003839656874734572, -0.20904648506114049, -0.2008749781821089, -0.06350655587394786, 0.36471958397751264, -0.6603734992626242, -0.6696646374658353, 0.504365954178925, 0.3994788594709243, 0.01254013773406199, 0.14425656481376406, 0.13827534724232268, 0.023726859355328426, -0.3036259887196344, 0.011679579819892198, -0.1323825829681493, -0.05680639521846163, -0.16155164054378507, -0.21514717784220388, -0.2932762256856868, -0.25997154994603217, -0.32280112295327074, -0.423976948809485, -0.2947946968015433, 0.02974235556286447, -0.06208923101479379, -0.2793814462566759, -0.26690792421956794, -0.1994375480685585, 0.2513466280307661, -0.02288437259161938, 0.34527356265671444, 0.011547920943123683, -0.3212548229423512, -0.09282405919272843, -0.08676381825010353, -0.8357144572540713, -0.6515268457770377, 0.15875767182602388, -0.09792399527867326, -1.5519975355447257, 0.16635422485762077, -0.3419964113237616, 0.115570523689806, -0.41367832599169524, -0.09655389213347829, -0.5992894739150166, 0.36029526034723525, -0.41775272037916444, -0.005212039370063866, 0.03919868000252772, 0.2960832752337764, 0.08876376934590749, -0.2000438126248908, 0.2881965714255618, -0.9876364891091728, 0.0430582800377127, -0.08958136562123026, 0.0714223409642708, 0.13969391953882296, -0.24448911013832486, -0.08534816249150082, 0.027617400507028714, -0.20105118854577522, -0.44803137628359346, 0.169229067561372, -0.14343047967838732, -0.21809736307789596, 0.2718685825921439, -0.1827916927997978, -0.285756704345922, -0.6185397325213327, 0.2712372600340023, 0.18399069910505592, 0.37633902716336004, -0.07657120819433592, 0.15589589754798977, -0.08401302835224062, -0.07310078499557553, -0.33116504625167265, 0.1846405766262045, 0.011695216588097983, 0.07084815848499615, -1.1958554634200629, 0.04089522393583156, 0.02370975859766556, -0.11180000485369511, -0.15386707069132002, 0.32077087732994264, 0.09806782474389497, -0.2990921655413106, 0.12114560130574054, -0.22349747057141223, 0.1446596741720122, -0.042006898747055495, -0.15672136447295185, 0.10222726971915418, 0.05901144362041472, 0.526806068273518, 0.042573270528213955, -0.039029856601925775, 0.2276112201301151, -0.14286995817522288, -0.04830868939355223, -0.16948985717730755, -0.3345535261335228, 0.12632736844479928, -0.0867069898200896, 0.5303927271786955, -0.12067908338693929, 0.13528428053830016, 0.10439641883735533, -0.24678016171315592, -0.5745775277027961, 0.307231142845223, -0.020568472018367426, 0.1918917517966536, 0.006806952008157234, -0.1486489017146256, -0.02940414070379001, 0.21245597422961557, 0.2679295651095675, 0.006174438010320864, -0.5303145425537006, -0.003239361532186775, -0.06916812052814517, -0.22406717582112637, 0.2831343577860298, -0.4856460673139993, 0.2997078269214241, -0.07054733056321265, 0.08631939334166422, -0.2968713329752704, 0.18517520740811647, -0.0633580921074615, -0.359832145821809, 0.1455461839135357, -0.19856853898382482, -0.092935465193165, -0.17663403175983874, 0.08325204616226632, 0.027981627040054313, -0.08689096380813997, -0.06752170674600219, 0.1908509837572272, 0.09770770165315985, -0.27188535244557316, 0.04772135712574334, -0.028716999321171516, 0.14742542247961088, -0.21104817186680494, -0.00792020001391173, 0.10215876977185417, -0.44430563501495485, 0.3909855679801297, -0.30419704928249075, -0.16332663335364414, 0.8030832110258987, 0.5100400746045971, -0.3904993617158531, 0.37863123208314864, -0.06611813586892876, 0.08302405424497696, 0.9900813789358616, 0.23562782208215532, -0.04281395975261747, 0.1660419049610286, 0.09405136927772795, 0.3129966076500773, 0.4188861048764931, 0.007058149210840497, -0.22796855844198133, -0.06795478030554068, 0.09954963206439134, -0.41061352682392005, -0.12039654502029362, -0.16950565344449844, -0.22886272287906462, -0.1294239611356619, 0.1138046401538912, 0.030390747682822714, -0.24601417702597184, -0.3311269892608762, -0.05031329742718912, -0.2842702233785257, -0.11400768077639624, -0.2067365007064427, -0.30132006547626755, -0.09023593589952306, 0.011307721185075703, -0.24040933300621112, 0.10786942077231013, -0.3513506063863766, 0.042243284081225616, -0.07785777768158217, 0.2561837603609132, -0.545744736949953, -0.09966968082984229, -0.03948489375726883, -0.5772837188865636, -0.18117186312527125, 1.1332193685670802, 0.10637024651061747, 0.01868804728506714, -0.14724109643943858, 0.1434276324629424, -0.36089991224264073, -0.15745532229109735, 0.16887976885260933, 0.044135689665456126, -0.09850910418919986, -0.09607018143104004, 0.05408973983250777, 0.2729160402771384, 0.10356258609752676, 0.12937082123529886, 0.15581644017793977, -0.5017106363464378, 0.2723089574880958, -0.814204389763319, -0.1627457598010017, -0.4238134516921863, -0.5444545954387084, -0.14442740066264662, -0.014149824210539532, -0.40378359597527475, -0.26264160367135103, -0.008373199311269756, -0.07179536588298192, -0.19406464788098687, 0.004166889669655142, -0.24308194285148457, -0.259296225447701, 0.030619225520729706, 0.46849077627849756, -0.25910019569954906, -0.06525800509993245, 0.06600846451998765, 0.04188440396783683, 0.5122425411960199, -0.22131281282206616, 0.6915411344110388, -0.5602133122523529, 0.15116563613417275, -0.09932499739797693, 0.08742976874880394, 0.05051318816530228, -3.5790332581860294, -0.05915138290284439, -0.8934757165515552, 0.25743027012350905, -0.11252388462347088, 0.10715897455123853, -0.21162228796806695, -0.1631249364914, -0.2204197985102154, 0.30153080561680823, -0.5107261799879911, 0.1468613047390484, 0.13704462384326904, 0.07906300717945383, 0.26909553475125497, -0.08971921582145416, -0.7389725345582594, -0.045613478958857055, 0.1520128641465598, 0.060604606883397666, -0.043835838361554406, -0.4187183486797775, 0.23730312961388836, -0.14560980036190865, 0.20452589202693047, -0.5925035476785669, -0.04068995824962345, 0.43170644487863324, -0.051479224938531895, 0.2979992383193905, 0.04245911556773734, -0.17262635607790505, -0.11478278477472659, -0.031329041077497816, 0.2672730791422989, 0.04511947295296283, 0.15128400944008905, 0.31255674724183136, 0.07387226442985388, 0.32490969574526685, 0.2064699784206835, -0.021848196431278298, -0.2619693220792512, 0.2249137185959305, 0.26001815467971584, -0.2300109577582105, 0.11855111587289532, 1.2411375908833016, -0.40482872788138347, 0.06778809945656324, 0.015478878284821225, 0.018152430172072784, -0.07697511748316956, 0.11428474644835755, -0.06203201557939017, 0.06924957716016879, 0.35281634116182214, 0.04132622224984415, 0.15814971152471424, 0.2680965009575137, -0.38610878816318184, -0.04890570548937179, -0.20002006139625852, -0.1336030755645793, -0.13322189962530945, 0.11109698470638438, -0.4617591967729481, -0.10855304286843173, 0.21353722668181135, 0.6206702509706714, 0.32964232742855915, -0.26262436134481265, 0.4518059932146121, -0.07270428502667335, 0.5167149263462787, -0.07153418189476331, -0.04958910478185894, -0.34151810085608025, -0.2908099157048257, 0.8821095266943344, -0.6405577747297498, 0.18673721287896747, 0.5635256726314537, -0.22787381724875963, -0.22425660605367165, 0.029456589856124937, -0.3672913718118207, -0.11169601619311764, 0.3034101445512122, 0.027171057148079932, 0.2682750649186058, 0.03493774277704275, -0.11338523399666238, -0.04214263176571549, 0.2703593983267479, -0.47534851989494037, 0.014858783750147337, -0.08725403297718361, 0.18774204076929538, 0.042042012740397754, 0.8363441746998609, -0.4664035609069301, 0.05787276487260338, 0.04201965809120112, 0.3427104201110941, -0.21812911029403043, -1.1217077929295638, -0.3884422477635127, 0.0761771797840209, 0.37278872387968, 0.5239534752879957, 0.3760603894194678, 0.6548572761088194, -0.1729310760586511, -0.17440028827209172, -0.35881352329067356, 0.2668374006737169, -0.5736787324179484, 0.038231199252560924, -0.04863446818771273, -0.21645474808278248, 0.06942752222589907, 0.09456656554247292, 0.5344499890405658, -0.271193792075583, -0.18714529372373342, -0.1511382507345306, -0.15439911527029415, 0.13017290091728836, -0.03621863091343153, -0.1496883701914799, 0.11519949436923724, -0.04603715437828056, -0.19032987729049475, -0.20251753798297964, -0.11880037627974528, -0.08812582339914143, 0.1026086333487326, 0.005439194078727013, -0.9625800135395522, -0.36945487063277677, -0.23475448259123582, -0.4711500186804493, -0.03913439123612569, -0.3388959053881819, -0.1336411635025498, 0.08968935303250211, 0.05755904216001383, 0.2203914684668071, 0.15379727542401067, -0.018167090220563946, -0.25151551641857134, -0.017394289305825733, -0.06331994348924098, 0.18944811637121262, 0.13521343780597453, 0.607001266423376, -0.20287759565419522, -0.19090488708268166, 0.014872142039633928, 0.01566078119927954, -0.13075261479664224, -0.0657672152752133, -0.0766713899064255, -0.01787725976373144, 0.34134965900263964, -0.08697520117562298, 0.28385478786548807, -0.07165034394987738, -0.46389880363530017, 0.14395070742929292, -0.18922221087518853, 0.6139789736412176, -0.018792571771519443, -0.0590144194728873, 0.011253841505559647, -0.4996845701496562, -0.16522157673367255, -0.15433150062066864, -0.5292995783174278, -0.2057990780938556, -0.02068999470610602, -0.021022258127500183, 0.3845110144894287, 0.7693437065106526, -0.16851060549738722, -0.1945963346709421, -0.048872343687026025, -0.0026189532623842075, -0.14963598631749497, 0.2693485400023938, 0.007204476004781369, 0.13883993879138182, -0.018120386411069632, 0.06371917527249782, 0.048759534225771985, -0.03828415959524398, 0.46644186810993593, 0.18215754475061546, -0.3110216319615171, -0.2117330011985991, 0.21861509347878, 0.1406493159949091, 0.912829800518693, 0.22051822189574688, 0.04589853960351793, 0.35695319635789835, 0.054204420945611856, -0.040718790186059874, -0.13711935584982748, 0.15959171134551037, -0.1877393692651581, -0.22614678931957477, -0.343286867872232, -0.5229522527893257, 0.1269377967527434, 0.041969739740410975, -0.1875665967393186, 0.04374117861828749, 2.73344050384597, -0.015670486493764343, -0.0041238966858325385, 0.00354285524198608, 0.08861003196931334, -0.04779679599677372, 0.3497715030094618, -0.07883709301962538, 0.20431849825020687, -0.4640644301375298, 0.36263710506033586, 0.2822130091730323, 0.14926604204584148, -0.20585863101814617, -0.32906679205687184, 0.21773317668019473, -0.07564030343898509, -0.18810230646128967, -0.005378562494211664, -0.5851348404190971, 0.0625458963999495, -0.3818155975524955, -0.022585584833452174, 1.1100533611922154, 0.21379224653505102, -0.1379441472858761, 0.2545625059069225, -0.0849520136567987, -0.688071278393333, 0.4224919458450993, -0.1965920190539769, -0.933028704904317, 0.4003044206688403, 0.257626845755444, 0.5847673634747241, -0.3912313608951763, -0.5215521210957043, 1.1868080893548887, 0.4460003415382951, 0.26767654163249227, 0.5439515844826576, -0.004360891211010932, 0.3298457362409364, -0.23001968529326647, 0.4994422947354816, 0.16588349567068297, 0.40330359383657877, 0.1620017512839098, -0.08211929263227864, 0.38793141210044746, 0.6616244113536509, 0.13389793789211438, 0.10625748120022363, 0.14910229986126916, 0.20617523169324473, 0.32658124610074074, 0.2330884460957349, -0.15919652502192935, 0.12831165178755127, -0.1961484933088899, -0.021004528766925205, 0.07665063517466039, -0.07648176223810754, -0.09995725139649911, 0.0901848162754453, 0.1768145421069327, 0.08943632856891073, -0.07522403300340898, -0.02470992846502643, 0.06982642436412205, 0.05085427289044695, 0.47831517142149366, 0.3554597698717417, -0.16864298697571425, -0.3823801774353791, 0.03784929273812635, 0.35253677058251226, 0.03543793347096713, 0.053622977104473425, -0.4221346512617914, 0.12240043350959016, 0.3397219042872813, -0.21054154811000045, 0.2104510932096345, -0.21175697043807165, -0.21286940406643048, -0.09397994148202912, 0.013527892179917667, 0.37988170855408604, -0.063432355501577, 0.42779180561512803, -0.7013409501605319, 0.05402142785367131, -0.002439258054400798, 0.013918224709040352, 0.14339952310996187, -0.11208691006197605, -0.14217923456383946, 0.06670622965171215, 0.31720750559225336, -0.33090255249803086, -0.1258433449967375, -0.14631915815696406, 0.3826635215355113, -0.11413540310587865, -0.39501242154430344, -0.46839477928396595, 0.0780396197949793, 0.39458639064931117, -0.0075211860509371556, 0.36464265003045676, -0.36371211110771734, -0.21070988544857058, -0.3663612787040683, 0.1349011975344455],
[-0.973305761184611, 0.053663620327035304, -0.0858894104558316, -0.39626041591711936, -0.032705808676139744, -0.14183092968349054, -1.7513628299375883, -0.0811049355075616, 0.08263016124109597, -0.06916395185194439, -0.02319955802366694, 0.17013776287446603, -0.32271590628592917, -0.06125012360374863, -0.40402825545787385, 0.11908488442045999, 0.34088442373766314, -0.26785972167037003, -0.12206799600680511, 0.038757155516229796, -0.13356620874558692, -0.2962426894454984, 0.07709480212312905, -0.05358158488230386, -0.6936120421004474, -0.15820711913394525, -0.11163840577363363, 0.02888041686422937, -0.20730805271409997, 0.19359880484346892, -0.4818826238104541, -0.4155625140334541, -0.03668404941666221, -0.2638078731090559, 0.08995459517973062, -0.1390530063816727, -0.34214518306236863, -0.4948282518809856, -0.3854051994063605, -0.007011754708054017, -0.33187808544846215, -0.09282016525692974, -0.502907308736227, -0.033583761493257015, -0.2922312727852181, -0.17701140172789656, -0.04781871552754998, -0.14030027698518652, -0.7484790357915556, 0.09579220600624587, -0.11681780546306021, 0.14117508240514887, -0.31032717284220257, 0.034304054848238744, -0.04198631467869617, -0.9879447356990332, -0.3244897201439967, -0.08670755323427198, -0.23013169439912437, -0.15496314317435042, -0.02381689806031652, -0.3441165469259948, -0.39674263465744414, -0.20319504061475221, -0.08207543667820658, 0.02520359817513443, 1.0422457027531653, 0.4202688128142158, -0.07089036279097775, -0.19494348519424362, 0.037527754349149144, 0.07052967048460863, -0.5582198125703022, 0.015082605024739846, -1.0614574874013418, -0.04614830242632528, -0.030336005932363016, -0.12783771213487238, 0.17083182925512425, 0.16147727899075587, -0.46571826525047, -0.048524159059240395, -0.594791353156523, 0.4799732333372087, -0.15618665112960028, -0.03775998071771258, 0.4220158362749527, 0.029113937591446974, -0.5584323563164031, -0.06908581793994233, -0.353107407822763, -0.028959207468092774, -0.0825912800173471, 0.1647176651980536, -0.18853771460138594, 0.11446958548542316, -0.11816897328281896, -0.020235046039986597, 0.24281185046933587, 0.23287731129301137, -0.1783372030677621, 0.10299383148620585, -1.0005781016486806, -0.2334516258845009, -0.2264324228234101, -0.2444090490490609, 0.04996625896158548, 0.08410326910289936, 0.028313300768219942, 0.004422495791899077, -0.0531082959294723, 0.10129516532865582, -0.06956165131633513, 1.1364544832870138, 0.2267419917408011, -0.29871827796813266, -0.09568098495982628, 0.868998030723014, 0.2106468423456515, -0.19956869502059063, -0.1280414761010785, -0.17478132907510088, -0.1440896826394732, -0.23822897606229132, -0.08091525271505151, -0.0352577159305736, -0.17739734471502439, -0.031679458608181256, -0.4099419680215965, -0.25441536745264476, -0.030377962587270088, 0.0038397721528816713, -0.27722058364115637, -0.004445786548231781, -0.6076275685516038, -0.07097319812061165, -0.18919839348022183, -0.39064832992258036, -0.05444383547114538, 0.07807857986999275, -0.04814355688832221, -0.17222735178221965, -0.03043431626508513, 0.11008584764690418, 0.07058963181166858, -0.17740882946813488, -0.27975008717460875, -1.0074395305577575, 0.03035810712274724, -0.10959607014228534, 0.2768702306959638, -0.4944465643885684, -0.07257209639239313, -0.15791897980200303, -0.7588996425347697, -0.31107248218069233, -0.09107506000220691, -0.03872251802812813, -0.5311696354623787, -0.039788802511014035, -0.09629084473605985, 0.12303957311761496, -0.17134065256272393, -0.222070146936954, -0.08013355512285046, 0.11457642082215626, 0.08833552453931422, 0.024481418937175384, 0.12847193265595327, -0.3084207150537943, -0.2280714627661471, -0.23304674930072214, 0.5606068043995538, 0.005182434766777416, -0.022169257754225673, -0.350447730303974, 0.041021794793036866, -0.8653853110523052, -0.32838259838645706, 0.011778065037948312, 0.0017974616350042885, 0.11683483261209571, 0.44977395263681247, 0.06109264199766035, 0.09601876590556012, -0.0034014341077044418, -0.05387036246766599, 0.06648392913784673, -0.1392352854495454, -0.07889842413845097, -0.4612900903985633, -0.16306442262684798, -0.2649637515563118, 0.23889167327667502, 0.10697040406870778, -0.21791217634185453, -0.5599744181284434, 0.041728215079461055, 0.06589367421339924, 0.0040142639343007476, -0.5675574884701343, -0.15287245696912227, -0.03145129450368203, -0.15230818103914, -0.1959578271791695, -0.3167117758547097, -0.1889427525550964, -0.1744504946684656, 0.1452661807802433, -0.0880691808158983, 0.026589626858719774, -0.7426733475572644, 0.03674506663776447, -0.3332223521892306, -0.1623369717271667, -0.05063334226885421, -0.05527615013924653, -0.15805909281831343, -0.665358081633528, -0.15760109851362206, -0.38875478767794985, 0.2204914080481586, 0.20460256786000064, -0.17846333011306323, -0.20401997352983445, 0.07941852895782164, -0.18196591233396409, 0.19835496563915694, -0.17996560841818943, 0.3172022086187108, 0.08225186917179286, -0.1052857651703165, 0.19625802590891145, -0.02826621471764467, 0.24542143089405796, 0.25831108741703107, -0.24269861016783972, 0.011229076718542398, 0.3319480623783386, -0.3550730421400307, 0.1138116508182585, -0.2813321462694234, -0.32609447554219767, -0.27786383758964855, 0.11511646387268079, 0.03887965723071839, -0.16199190462696386, 0.05590932219573413, -0.3917018846280829, -0.245761537210606, -0.03072257126945109, -0.5113068456618809, -0.35858994465959115, -0.07768141818385674, -0.10733545313805226, 0.06824962958936893, 0.013421840876682597, -0.1258763145360073, 0.8238870997357607, -0.06760994587399276, -0.860018689124695, -0.300854498348988, 0.009912351510041781, -0.21027509254971638, 0.01988524061463758, 0.13599152582383697, 0.37169007880310867, 0.058441100390596176, -0.14631678273059226, -0.2516419949731667, -0.13221307763331866, 0.09009250079982148, -0.07998773875269169, 0.048598601140765, -0.1180543975648853, 0.013984378199450389, 1.0603228384344328, -0.06586001570947536, 0.012191676286804282, 0.1904453252131341, 0.05216673284894466, -0.1960504896706455, -1.1119447827902431, -0.03379030555180115, 0.20863054077443188, -0.06062923029667622, 0.4436058493934554, 0.4212311645515859, 0.07028104752561945, -0.08929267139348575, -0.04851863761243187, -0.34504943094652235, -0.44174555634061946, -0.2223240626367368, -0.041801827949936615, -0.0545830098394402, 0.15110558857241885, -0.028687760167740436, -0.40329220261817167, -0.09630784598070316, 0.010151985016005602, -0.10006232608915457, -0.02658401729833609, 0.06393291954765912, 0.013164934542085933, -0.26607689489675485, -0.012458815277935588, -0.05626632529275807, 0.12641929511820962, 0.044103074883830494, 0.12332379873701921, -0.28758323301743016, -0.08868486879440408, -0.016485086935451462, -0.5347081116009377, -0.08848319647984326, 0.050917087824095805, 0.004643799667824474, -0.2065731875692315, -0.23234685915817932, 0.7054359783887594, -0.12954177660254182, -0.056997288741216824, -0.038496124004189565, -0.04486927381471838, -0.08235048591579534, -0.3407266542717498, -0.14992064872441638, -0.13461002600912741, -0.20704263062369474, -0.05405641833363102, -0.08356635194161044, -0.5888367612429772, 0.13123064068070175, 0.03876364352346715, -0.020003123021860553, -0.2593187049360547, -0.271033784321538, -0.02548051783631784, -0.21549062993368648, 0.154331119404087, -0.26942026244577466, -0.2830569435503965, -0.025126060028861412, -0.33168271963640555, -0.890053561757748, 0.5222712351012954, -0.16735676452841386, 0.6060097327305548, -0.13413200540640877, -0.08037337849747925, 0.24358381342235155, -0.2256752886322122, -0.1615345172697608, 0.02981805637727853, -0.10831890820157664, -0.14607703263538696, -0.29923739348795564, 0.09023660784315315, 0.385539893782859, -0.06583512079717085, -0.18897050151037592, 0.06721256950293872, 0.13805607518974763, 0.09868942570011205, 0.12465429805353195, 1.131372119606087, 0.14537359160502605, -0.47878776832512165, -0.19113876484517117, -2.6083418864704613, 0.020867182809740106, -1.2692643548148645, -0.023234647054764742, 0.3423432306068091, -0.06426768969762127, 0.04089736315249311, 0.00410396575243409, -0.05698947418017787, 0.3528608753450588, -0.32129445089338765, -0.20507392312936076, 0.12182842038298915, 0.4865347341501799, 0.018491946600761075, -0.2117036495043801, -0.1497376050196812, 0.0030704603253029596, -0.14615429176846534, -0.09841664470933338, -0.41634788402414674, -0.16806111381500974, -0.14304504506382038, 0.2002215093733588, 0.14077503376170816, -0.1027350022096, -0.001990181048308455, -0.0805751279037864, -0.0673350886034738, -0.13797135238505984, -0.6075512897364371, -0.17131476434357998, -0.24236724854032157, -0.041097876106682085, -0.24057026322388952, 0.6919273450465224, -0.23548191957947123, -0.2618176129096886, -1.305165336734632, -0.2913231469208346, 0.051642055643137355, 0.11620539536154703, -0.11059145070029129, 0.009441709350717567, -0.1553112816562681, -0.27463114300172703, 0.06295667495907335, -0.6426876890851715, -0.07350296015181045, -0.21436195743867, 0.1260927268833685, 0.07807161094124804, -0.1573090766866893, -0.48813937406802305, 0.09989066120351464, -0.4176117426617354, -0.16941913862628288, -0.33497372034508316, -0.13346161024393038, -0.10446581600369341, -0.2758614416615932, 0.038611899551566034, -0.1150447564604113, -0.07337975472195327, -0.30714515650964097, 0.025720603762544827, -0.34920789187003165, -0.20244073481916228, 0.01999087613761935, -0.41759004211576084, -0.19298524404186743, -0.27703324643980565, -0.20894922290066728, 0.6747879918631641, -0.6788530735745133, 0.35288336541654214, -0.064326055765467, 0.026016097684447313, -0.12167124117188914, -0.7337290707952239, 0.06869210455578711, -0.0680237150166834, -0.2201933726526521, -0.10301202487058947, 0.10868999713733132, -0.12394369753590805, -0.808142924256221, 1.3953642344357273, -0.10597881674547943, -0.17125812239787316, -0.10170910902199783, -0.1886093780302855, -0.23448437369362998, -0.02013674579120297, 0.03843782303099099, -0.42636995136964856, 0.26945452994812824, 0.008399112517831726, -0.04925660666788189, 0.5815249174815469, -0.4692835381290847, -0.09285461333332418, -0.17699151549786496, -0.02642724241784803, -0.275545499856536, -0.0035965972570156306, -0.33419143020630226, -0.1880797942843037, -0.003222911048108909, -0.3352367792882618, -0.3898952039653605, -0.19939786446388952, -0.2556151918471163, -0.3897764740611305, -0.04428010080087469, 0.33979746868200056, -0.040915643528395024, -0.37694483126316375, -0.22617046781030695, 0.08331483942973113, -0.07045167407150621, -0.04306511792178271, -0.2748536949895718, -0.14377953818790834, 0.47860556538949667, -0.3572130853393506, 0.6510935974390384, -0.05883989118823503, -0.08858605786468757, -0.33901600328570825, 0.2306883905605939, 0.5210114702779677, -0.2120483029419524, -0.10143630497526848, -0.5277287347747692, -0.08575666437974436, -0.13204214934784797, -0.3232602172718051, -0.9094054977494782, -0.03457416186585427, -0.06534336889788575, -0.016216411192958375, -0.06293645428406233, -1.0778228547420508, 0.13168742939605838, -0.11189333530687351, 0.3273276283409765, 0.07834229260018528, -0.28442515309724203, -0.08438417544168128, 0.40967983484798864, 0.6688668188403674, -0.46335877455653557, -0.10028902157119331, -0.2187291620893171, 0.07162418431782731, 0.053840452414558995, -0.8274680745217535, 0.2675112061009316, 0.1100535522875711, -0.021075350423764048, -0.029025364027233814, 0.10142079495361607, -0.23064197082023422, 0.04324743670934593, 0.020318915245260697, -0.08638885271977821, -0.11661318931029786, 0.039632918534086785, 0.018794803935364805, -0.022742286604462966, 0.08026864301997241, -0.07009791538290719, -0.08658279749168861, -0.22167199810573482, -0.16943512021556179, -0.12433496084487668, -0.15255509786520144, -0.009927424858802308, 0.06803619191021687, 0.3712882299217773, -0.017198731734821636, -0.032571755028088116, -0.14457742276255492, 0.053584982836131084, -0.3265543055342042, -0.10303989838686603, 1.2563995175955731, -0.3967530272679573, 0.04062772651170382, -0.19438622404891312, -0.03574947090465021, -0.15456845262659452, 0.03707419875575593, -0.5071574698689896, 0.03459087032595611, -0.1799517231250309, -0.17020711486582368, 0.02257916694556377, 0.17264833234656166, -0.02763388341118462, -0.14324968419176293, -0.2509518728350522, -0.1590079818909045, -0.4733654406594797, -0.0010845551524016751, -1.162424372134566, -0.3893503942811259, -0.008430277958463808, -0.024058752191509544, -0.3231115700976102, -0.050086956849405985, -0.28873707658778575, -0.488532658210322, 0.06550619729203475, -0.14211198100654754, -0.3812429742995796, -0.015218372506205821, -0.11110785243536775, -0.17348083932225675, -0.308912293252371, -0.09441026830772435, 0.04167794474769855, -0.05814054711177735, 0.26969680385476175, -0.8688414165187605, 7.964170849964966e-05, 0.09232625570642021, -0.638674927029704, -0.2460123957720776],
[-0.4594425270803383, 0.31995413145879115, -0.36106824234380064, -0.6650725997295723, -0.14080934677846793, -0.1892607781506785, 0.888711704246603, -0.04623067405344522, -0.03762133048305558, -0.11399851740278774, -0.11598825944892367, -0.050840167800195596, -0.030286864389050365, 0.0066276626124645, -0.5596117495551178, 0.008130200298908484, 0.2714629737356772, -0.01129321394585767, -0.39949295849711264, 0.23961653593783333, -0.44008579135641546, -0.37909995213348063, -0.030425312294865187, 0.06882510772055768, -1.3666598679769248, -0.4167267001672003, -0.09041163847274811, -0.05500232315261671, -0.04149706408259002, 0.36059748900677185, -0.23090601236138483, -0.5131889761541224, -0.37752172536594, -0.6316078812153741, 0.08521822120636027, -0.09478171453474761, -0.36123033140527927, -1.1935953210252832, -0.6458480674145762, -0.2766556099811266, -0.5786335536632475, 0.14106004241949266, -0.5738044415130801, -0.00989566231104155, -0.49026809366432855, -0.26817555952720296, 0.14613926478696548, -0.23624182958684434, -0.5136418199882479, 0.8284461827815321, -0.7318095251356982, -0.04278521286263568, -0.4861783886320532, -0.09770699926677665, -0.033072741195272176, 1.6260175980235634, 0.026078219421831618, -0.19085423821022754, -0.17754767401357294, 1.4585441708007343, -0.028696178075610524, -0.33928963951833135, -0.026039990148717638, -0.2116446379328076, 0.007941886722972694, -0.3806158182312793, -0.3407323374058646, 0.286281653674159, -0.0790089626043442, 0.008172792827208139, 0.013548677988231088, -0.09644055168349738, -0.7417780263533463, -0.1384684328184214, -1.5440112242296287, -0.23688948050954448, -0.4418469419300022, -0.1305189896599501, -0.3943064914169652, 0.20559373228550187, -0.4379957518056098, -0.026106290962373154, -0.45805826203346606, -0.20750260151406077, -0.11619402622352147, -0.16001699892052443, -0.6574215491941343, 0.017873437608172903, 0.7617351057086834, -0.3059121019338754, -0.3482195265853287, -0.20302393304370334, -0.26556152502040464, -0.19029562537804376, -0.4654929164853496, -0.02603366014870424, -0.10407232783509213, -0.04855295690903644, -0.05701257975807812, -0.21995974092933027, -0.23518926800251597, -0.1974116537342427, -1.9998705995825794, 0.09132284351824048, -0.21354258959797012, -0.33387948303397486, 0.046388327276173154, 0.09975372656977069, -0.15483834827668158, -0.1827879320314317, -0.006596146134706416, -0.09876057096317277, -0.014380594740953604, 0.12022938350488786, -1.1254353320040136, -0.29974563006720634, -0.026226054528031152, -0.6457040957750008, 0.08100893574035593, -0.5743382229333958, 0.1176159801762298, -0.16636431682678665, -0.11510314462850169, -0.164067937745372, 0.12522973563351264, -0.11792767792477936, -0.22608082311029695, 0.01757914180456241, -0.4812917232399073, -0.2337148366396883, -0.037478141308090185, -0.5196185976874521, -0.29217372721701185, 0.053319215132326375, -1.141268888686715, -0.03322154491714943, -0.20697672504185657, -0.3958057907220392, -0.053476458816687125, -0.14663091447786034, -0.10769728467025108, -0.18177459689135259, -0.17754983728221707, 0.10209558614011395, 0.028790082702154555, -0.36972094530663907, -0.23458975770770008, -0.15291969112391982, -0.048809224968643386, -0.35343698150021985, -0.044084092648306324, -0.49811443128991373, -0.03281885368778697, 0.0866032976014424, -0.9575645670003476, -0.30716366421284347, -0.19518506799927018, 0.13515613268426627, -0.21379152382033026, -0.13814144904601136, -0.2467541446737609, -0.2771819467159248, -0.4589500291176241, -0.07348873108058705, -0.2906853099151577, -0.25248368133733484, -0.13724207552476536, 0.10502314146983419, -0.07077729752426903, -0.18995393674948216, -0.29324975046601126, -0.1977030543878581, -0.13727476287916357, -0.0773088818862081, -0.0012179602387532044, -0.14399301989923913, -0.12104920586293598, 1.091149347745409, 0.2682472460793185, -0.005378596557166024, -0.01783118553294569, 0.008197755891290901, -0.9393158276411163, 0.03297305180781741, -0.1249628199910089, 0.2022651530181282, -0.1761740072086948, 0.08978401389432304, -0.1522833305854194, -0.176050204604255, -0.43625867688873377, -0.06564423801272519, -0.3079353823278352, 0.019232390821900053, 0.1952449564669851, 0.04703565986643773, -0.28766968792229525, -0.18509346612044758, 0.0036964025093391595, -0.02542132363187963, -0.5672101300863536, -0.35938001809160175, -0.19478697757288949, -0.1528198663859868, -0.39732699529075566, -0.5310171924058159, -0.08719243673500804, -0.00960252178069566, -0.14601892168733852, -0.3836821171280304, -0.16309605209703154, -0.9413477119292124, -0.406664767558367, -0.37971051132160394, 0.5542929229972737, -0.19382685449108142, -0.12548623974349848, -0.646152172570716, 0.23600116561248974, -0.3340633448611681, -0.2669861971847471, -0.10168353851437961, -0.7392275209450107, -0.09828508578784251, -0.10607602484295742, -1.105667080488086, -0.207826882483805, 0.08697563688333643, -0.2691687183523531, -0.22718612376750108, 0.10784148101088546, -0.32481921051756807, 0.14210880227813313, 0.00025273039130284646, 0.004321691255133961, -0.2620845823624458, -0.19936991393298073, -0.13103499133058882, -0.5153803857928704, -0.23699697142567136, 0.0014334543110050733, 1.6026350105215033, -0.3837567724868719, -0.12188474919297855, -0.27837319501298696, -0.261348184168035, -0.05087103859210437, 0.05299784528255303, -0.38173644838913473, -0.2774580705311128, 0.1192283566819994, -0.2740372133934344, -0.2899534210996488, -0.05388149056477077, 0.321520280298162, -0.017966861318873328, -0.3752330219706698, -0.22478754122344435, 0.4932468111824017, -0.40806281108762754, -0.23516894376069256, 0.25416410773942305, -0.11443719470261343, -0.16480176552362902, -0.08172983797936885, 0.06524286147766804, 0.13524400819580934, 0.09171418684354188, -0.47587753444774705, -0.5695584367953982, 0.18676255060146885, -3.3804409054469886e-05, -0.20582287313470968, -0.0938642136014176, -0.30302949914430477, 0.057572897388453004, 0.06851725658359507, -0.21060127553294153, -0.09595662759084261, -0.34851148971592477, -0.05470413822333618, -0.4436263106271601, -2.0092486062933235, -0.025876706116630926, 0.6927899316255833, 0.17518616321709166, -0.19081555322574772, -0.15819112273056277, -0.012537688342990808, -0.19965869795211943, -0.10945055339169879, 0.188797303877827, -0.40736955187751184, -0.22954025564831398, 0.11264908515432301, -0.06782572448833148, -0.19838728444057416, -0.17677762007244927, 0.0033540471246925013, -0.19343912151064152, -0.04337496490888218, -0.10838933387145204, 0.01860974236737298, -0.00715542222671501, 0.14726081314594183, 0.34467706020962163, -0.22475489266940804, 0.01762450209090875, 0.18210871397560593, 0.13267369052680092, 0.3416593379210137, -0.38820538092646945, -0.12859033566710434, 0.38948874522641547, -0.6365124420357715, -0.2561464871584942, 0.03477943523235871, 0.015841299026579665, -0.37042121404018497, -0.279029949851916, -0.18184102678822983, -0.4976072441135149, -0.11543782227324782, 0.1010958676639246, -0.04029506396077536, -0.04834674263338012, -0.6413164443549927, -0.036736490872135194, -0.39491386088922725, 0.05690584968382279, -0.29738149379586526, 0.021025036201285704, -0.8841523308136212, -0.10592859623289032, -0.18552124594296815, -0.040950577615823054, -0.28913326937031547, -0.7035794300581838, -0.08091686506854728, -0.012740540072978791, -0.5952823916909055, 0.12020024103765183, -0.3575857386920368, -0.41142518993490657, -0.08620265524118378, -0.04764797976235172, 0.25788928898908603, -0.3652566389291493, -2.1770566909923583, 0.406949207140763, -0.42225798358342914, -0.014263732904481486, -0.33425770620856426, -0.22747574350698285, 0.08536639312577783, -0.2733315143679832, -0.039623267214963794, -0.39982489139575234, -0.025507173106344518, -0.05625472167043017, 0.015174378709001426, 0.25486129002277463, -0.03804733231773147, -0.1809782054182348, -0.016932445046482496, -0.06555616909265903, -0.8012016528513765, -0.2509224241171903, -0.6330578939769984, 0.5553417563865587, 0.36407390417016644, 0.022742793608440427, -1.4962421836934183, -0.08159303337673879, -0.18512499298020607, -0.08406076085403291, -0.08091495202664374, 0.33569814092794986, 0.028152113573131406, -0.07860232273690745, -0.14638587910188877, -0.05042383833809917, -0.009779838098825908, 0.24367077897150904, 0.08597308310542212, -0.022408600019450182, -0.292691596422064, -0.11211077775593829, -0.03075301441473298, -0.3507929288337512, -0.4216638413577932, -0.12112915484694131, -0.11460310124474457, 0.07658934568749923, -0.46679060245134085, -0.07549039209335186, -0.052762563433161894, -0.19712997762143186, -0.1705916660918726, -0.21667623259651322, -0.41193215400902844, 0.005137438570118746, -0.6106799512566831, -0.14540043779474116, -0.31436371397137863, 0.3971159526538152, -0.18903334026135257, -0.4932045124475227, -1.3669324437812531, -0.17233009174103353, -0.2518074570915371, 0.03759348963242307, -0.2114636333816059, 0.12522839555592385, -0.09676614222724134, -0.535113768935523, -0.06328570263290588, -1.360343448619798, -0.04061575324743876, -0.21612976396723138, 0.12195395755209937, -0.10411660739622626, -0.022690984944069122, 0.21993238724439443, -0.14605374788045636, -0.148533989043835, -0.13784151674971498, -0.466459163004408, -0.12426396671049714, -0.16004398402395806, -0.21657437224678147, -0.1824948993771177, 0.245883766500627, -0.07916057071979136, -0.1719952884866422, -0.13084397188382665, -0.21942524107069147, -0.11394210422030364, 0.005808007636606404, -0.498947248498412, -0.08818808816606698, -0.1820200015053947, -0.3029435890464624, -0.3707205136623154, -0.4137394582085582, -0.2806474368373404, -0.27833419138376264, -0.20233980484776343, -0.3533710440952817, -0.37018053498748366, -0.18237049956314347, -0.4248862644402157, -0.2956303641973214, -0.19698956404859877, 0.0046608076390236355, -0.3605163967210686, -1.105351848760218, -0.1603792372448822, -0.16632077020866506, -0.3408796862588326, -0.1658983186298331, -0.2848711741351566, -0.2493420384335769, -0.10805236057448285, 0.11500573635025177, -0.23652648951567148, -0.04941364203178111, -0.026540364867541504, -0.14675291133024046, -0.12933743026257827, 0.25582912794492324, 0.008868765406887797, -0.005167520735597225, -0.34739464298392125, -0.3414973526567528, 0.19329471945331725, -0.2239512631188848, -0.1634530742765776, 0.7365735476330649, -0.3462817789916873, -0.7634584473185103, -0.2619071988274218, -0.39409159138170663, 0.11616578341936724, 0.051701673766856006, -1.3484041222546723, -0.06426627880293798, -0.383062739930725, -0.06853727053376385, 0.25126982320980684, -0.5104521507945261, 0.04748920855630983, -0.3379849142161194, -0.25756568064297575, -0.25237503941936273, -0.32494919275104023, 0.11268752191718079, 0.12170779639841438, 0.1349529076623418, -0.3279479451294823, -0.08113594776314034, -0.41442610858490886, -0.18169343339971752, 0.013973704337343816, -0.47569791981526865, -0.19478268655605538, -0.6663042814227598, -0.184633768253836, -0.8177139235868935, 0.08112054527706726, -0.20237311093156865, -0.2865676245581756, 0.07811471618066756, -0.24974672453708746, 0.07942472656813286, 0.15909101994942812, -0.030224441414548654, -0.0069635431511241306, -0.3578426080222563, -0.04240649318141394, -0.14713349943887977, 0.12076115080568768, -0.36535136925636963, -0.19448881143554866, -0.18295332427027247, -0.010897180186062939, 0.07955019720283221, 0.6800040069059129, 0.08535225494983101, 0.035027506254427125, 0.0479620294485618, 0.01800570925119954, -0.08520956797056094, -0.2659024840083112, -0.12662391908028722, -0.0034845729728606735, -0.005660312315319656, -0.16628374625822784, -0.018476813014407664, 0.043343402937450384, 0.29384726834221714, 0.06556137039643663, -0.2263941636870881, -0.22673038046046323, 0.0176020187976944, -0.20381870360276366, -0.2818291823914537, -0.3506067765314549, -0.3543204630900943, -0.42428353775422545, 0.04772358924048659, 0.19704991516476048, -0.16530202120759221, -0.445636714358895, -0.015459877163395702, -0.3729525751562916, -0.0436610186713828, -0.6896831609096961, -0.047426108091987246, -0.3231984891180492, 0.018281855062210077, -0.10292201462370969, -0.07460748147460639, 0.01791254512435548, 1.837516790409551, -0.018766649716984652, -0.38392324780450804, -0.14504111913160112, -0.09890625728377264, -0.5262180480349816, -0.12917338914603158, 0.046256495458697686, -0.3451131056012393, -0.2991314081378442, -0.5700173776440929, 0.013862230675819246, -2.3060940729062605, -0.12544941741422044, -0.023829748300874996, 0.2820056583913622, -0.1740107638836584, 0.0019848998326361346, -0.34817813052408275, -0.26480626512788025, -0.03180281478918746, -0.14488321117226338, -0.041090343144897505, -0.1262019341182863, -0.11443738966108892, -0.32559187576826715, -0.1502570181468293, -0.21278828919388149, 0.07832428794312975, -0.021664316582985725, -0.061077788050711714, -0.009462412122416764, -0.18195194137549506, 0.08790858148504495, -0.6557636201481858, -0.34262859793193823],
[-0.24921155905660416, 0.23476179373468908, 0.0015106550647994682, -0.4080087337145137, -0.021460414265768054, -0.13619126685605332, -0.34369685404092987, -0.01662517954755542, 0.026167640945613087, -0.09739882051940091, -0.1429808921773512, 0.04216910544215523, -0.6026726966765424, 0.004168321935056445, -0.1983821343851424, 0.027099602228071135, -0.38877827274289484, 0.10492178127579702, -0.17122049835480996, -0.02642780858571861, -0.2951727978597298, -0.1664401377321196, -0.0502404365669096, -0.043409287785750546, -1.2137434404755483, 0.3137164067813827, 0.12215701224207307, 0.027690307182001283, 0.02810173120991296, -0.12639693154554718, -0.19916104125830517, -0.869911214561349, -0.4065114480450804, 0.7900338517815045, -0.07173087853714044, 0.09835586408344288, -0.26676545626213505, -1.190599082416213, -0.17624292977617292, 0.012725934883907523, -0.6362383201086182, 0.03401596304784358, -0.5488971720555411, 0.01942386349817266, -0.08459271499569045, -0.1974988175317129, 0.2503242369120138, -0.3507613751657567, 0.2891515180731828, 0.5430727723133022, -0.315695239365284, 0.023174677865145033, -0.2675248180088809, -0.2381472735179811, -0.011363469429087985, -0.1533249133893259, -0.07080026213207723, -0.07466067840902768, -0.05164016537525773, 1.3226100173843107, 0.02602937717866417, -0.3661275988338978, 0.38265276343880483, -0.06915822180446939, -0.008055584309887159, 0.20786494909895495, 0.09453288084541601, 0.2604165279034626, 0.1827341027811381, 0.037847480668372575, 0.09755819048973015, -0.13841023744683265, -0.7253356853806484, -0.010785834953323902, -0.9940430020934332, 0.08570522724989643, 0.1474723314863657, 0.39302026135783774, 0.0256004326111037, 0.1034562792962465, -0.47550152761783604, 0.4805190781552507, -0.137666421277179, -0.4400169807699576, -0.18506529420225382, -0.19912543199557395, -0.434011682878323, 0.07612152227986867, -0.540529316111307, 0.07564302276163969, -0.1584133241009817, -0.11709503637640277, 0.004798119185160639, -0.13521034211231567, -0.061233991779819585, 0.29569188544196573, 0.009333812080277823, 0.12494905757022963, -1.043811755286849, -0.7845530073477903, -0.04211880040666905, -0.030993242125558172, -1.6338971263567896, -0.010729168878941566, 0.3772272795236965, -0.31151309510553404, 0.10074236766307793, -0.06566191608554203, -0.1656519852495099, -0.06849545548873583, 0.023396419593324228, -0.3547782915789159, -0.04610594539846653, -0.12478566750727794, 0.23239078273369643, -0.28848115704667904, 0.011305715045842291, -0.7272028713613414, 0.11908859623269609, -0.2039726096679715, 0.016413134873196313, 0.08775522005575895, -0.14291352464597185, -0.0321968589144657, 0.0190143296908182, -0.10555320270878268, -0.14669593064974193, 0.02524448975007352, -0.5310314595658853, 0.0027937384481041263, 0.02833151359786047, -0.45454830361359105, 0.1971007490945292, 0.20776518342042072, 0.31745295529059114, -0.028087321998616546, 0.02016862981680601, -0.03810203153533776, 0.0492211510154784, -0.03357881115284473, -0.07150914567237228, -0.2766479135106578, -0.10823187406518878, 0.10468385376479887, -0.05664547377468355, -0.1356483910333172, -0.18901505865490573, -0.2085781491237593, -0.07601453299892869, -0.2402809255351535, 0.1170309250653653, -0.4910166767386517, -0.07180142028959594, 0.026536456178517413, -0.4202762846634526, 0.004102833341180628, -0.26351499503204207, 0.010097735839789377, -0.18356474244218385, 0.1280515371468887, -0.09456923608530884, -0.18007968130711202, 0.05657174456480864, -0.22947114776534888, -0.10089884808788095, -0.06424644876444605, -0.10733866379370748, 0.05730518865844386, -0.09580901732397491, -0.024060627072243893, -0.19044170938383984, 0.03402219640155132, -0.854500945264535, 0.060629880083836726, -0.21562505016145636, -0.3609100415506333, 0.0028585901065646896, -0.954970171975972, 0.10399139226766366, 0.044154006546097045, 0.1260838335092852, 0.16020672754498508, 0.37566996296951205, 0.17430063841245266, 0.017854301649802156, -0.14581406929484653, 0.15466787524022077, 0.25284010367751575, -0.01767592114720331, -0.15333673249073, -0.20514449033400836, -0.15269836017124894, -0.2439946254692137, -0.02212157502009254, 0.05460302849338756, -0.27813673577489945, 0.04472611754328804, -0.24645838672607331, -0.09553130572674778, 0.06367768039459744, -0.35934386461832046, 0.21787982005930434, -0.015686747466881375, 0.1087745937801388, -0.12775935314751388, -0.3416497196588184, -0.26680183481730146, -0.05418789249476359, 0.023492106265700113, 0.11088398049323261, -0.10513475827852606, -0.9413303417846866, -0.25770355612254986, 0.3244414483195954, -0.20415071335668716, 0.7052018239314041, 0.22078891199832457, -0.45892063373922903, -0.7926302016363227, -0.05866199383061784, -0.3803938662837602, 0.17904829895928234, -0.2804402626979506, -0.05318751168070993, -0.1324978760634558, 1.0225752720604566, -0.20902843864845602, 0.07069070798379717, -0.10867512131623894, -0.037856299625975254, 0.17593954500152753, -0.08341201161707815, 0.07087423554838689, 0.02074254467458696, 0.15603537607261375, 0.43537685513135604, -0.2994358814268288, -0.1097518642592353, -0.04925405758388851, -0.24032044889062684, 0.02363127666236656, 1.6554495236282196, -0.2104501637817474, -0.30621294970032215, 0.10381529726272919, -0.0062831569834869636, -0.014632594709993719, -0.4004929400299712, -0.584383140142684, -0.12191034612403982, 0.43780952779626464, -0.41248464424888376, 0.03498676670185043, -0.016492886353714622, 0.000835377655256244, 0.0881067077670621, 0.06602562969630453, 0.03250029512125642, -0.08799602906335506, -0.13741531987473668, 0.11821454100233708, 0.7104155516278745, -0.015922972054542742, -0.11548087153314801, -0.05769495284394544, -0.013434390056756292, 0.47207254006226834, 0.03487226972615604, -0.38073345155784066, -0.1878380114133364, 0.10619449953399981, 0.07886326973256953, -0.1865508473365867, -0.13942097882785243, 0.1342187703588062, 0.05147240800018464, 0.02019505606947141, -0.053660613966343015, -0.05550974666561372, -0.22105075670183613, 0.0731620931028874, -0.37597818643695435, -1.6100044521009125, 0.13025338865928224, -1.0677459807256489, 0.2726493542401491, -0.3283391639027884, -0.05226487516826426, 0.16067228230126973, -0.21745174228700556, 0.06041658666724181, 0.3459766078858854, -0.23255855375319867, -0.1849387380713363, 0.16730282371733077, 0.059383470587648104, -0.07908831939866832, -0.15665708619674695, -0.12331383012342204, -0.02196364346805574, -0.10940563600211732, -0.21033242951376474, -0.005001277444560683, 0.10838933923823071, 0.16954970427188687, -0.48825623815134234, -0.10791221169720848, -0.20715071045906167, -0.008125514788926097, 0.19074056647431736, 0.5193274074516857, -0.45528252019160986, -0.20073778536606623, -0.4214096200305452, -0.6815472968898596, -0.1862847355672827, 0.10267041860276407, -0.06063703432493816, -0.059347147312515856, -0.28836732379565977, 0.14585937238297703, -0.3244544966562737, -0.08448288216914364, -0.09414556037566316, -0.03997246249135523, -0.10938453523882473, -0.2682127580496751, 0.025876496004247056, -0.7514075719761819, 0.2569352996708282, -0.25781124693124435, -0.05678691721599264, -0.4705080876227766, -0.1251219556115939, 0.12942586393991334, 0.08874534961655262, -0.0336320270414815, -0.254965687454626, -0.03402858328426127, 0.005814912698180073, -0.25042203070514, -0.2844071174530222, -0.08539986897444235, -0.3573179665005781, -0.045185197008064344, -0.7704973496083193, 0.17479601070971912, -0.3585855081504213, 0.3900347719158567, -0.10556280177616909, -0.4497156654701398, -0.002126271970333376, -0.3518306390098053, -0.28221804880587503, 0.12234397853469269, 0.06635568120650809, -0.13054993366408726, -0.280729598315385, -0.048264761135141235, -0.811524947049777, 0.00858482435572046, -0.23006903502126488, -0.03225443713605295, -0.05537397667218843, -0.012827903997804318, 0.13881614932090636, -0.29062469350130976, -0.6083262304495561, 0.46190625416817305, 0.18712715060463045, -0.07902690408148615, -0.12883940444289207, -1.7485566053791959, 0.0785686109416269, -0.5650656537562306, -0.01578610454595547, 0.008042935279690163, 0.49429038648757107, 0.06398725819223838, -0.10509212923381281, 0.001223318552253907, -0.07452033172485156, 0.026280361679592794, -0.20666697408558168, -0.009535440250902312, 0.07676036265411769, -0.4087784021534768, -0.07916455379303042, 0.07048809224306703, -0.18979303451204435, 0.7624129545160961, -0.15600392100674493, -0.1881982174179912, 0.22566246146679936, 0.1332338006614424, 0.18067665462732846, -0.1482584900337927, 0.08018202780207413, 0.013610349810921189, -0.18553477245406907, -0.25129334774459183, -0.3016967821709841, -0.1496912967490575, 0.01227744904410453, -0.010070115440460926, 0.510662894624466, 0.02133798068929381, -0.1566842570458303, -1.4308330414106039, 1.0144777393250985, -0.07824090034212643, -0.01348772140847698, -0.10974388403151769, 0.13700183142255737, -0.051249356116478, -0.643239537524817, 0.09049908307511369, -1.4920518721977496, -0.03272007419387742, -0.07067370840252916, 0.12039871465105303, -0.18588856998060013, -0.13621622743966605, -0.03157347710616112, 0.034307742268968475, -0.0452431663204174, -0.06080372970724986, -0.11532816475854862, 0.028996220784188707, -0.08842480996992136, -0.40400178619213395, 0.0718542135667154, 0.04948882929542555, -0.12956819802394265, -0.05445698727181012, -0.2710918237112777, -0.23894321040824085, -0.010683575803345065, 0.07138632464617167, 0.046779077013021725, -0.14265825322108067, 0.10621615226880016, -0.20978035032548242, -0.588374227927185, 0.5753516719445083, 0.0950775763543578, -0.3672865854141862, 0.03394216595828036, -0.059859834126529864, 1.5959412360390244, -0.0699712847636341, -0.2350720232562711, -0.22075281188016868, 0.026986929778607067, 0.10638311165637808, -0.11499644133747977, 2.6903008666479247, -1.1980631408375022, -0.06590856096190581, -0.26265932296358707, -0.12624468299902508, -0.017687827484736605, -0.15408276636876458, 0.1341396219633815, 0.07808242521582218, -0.29683242290122197, 0.12999846438213966, 0.23395731643490345, -0.03811270175047297, -0.7587946662605338, 0.45840835617337156, 0.02201997226364338, -0.060552357610857324, -0.7150691665759689, -0.296509473628105, 0.10444049159654784, 0.08784309301263897, -0.09958608821628631, -0.9993136260048506, -0.2855063273799181, 0.7741834159388952, -0.1704137124177398, -0.07696575037720799, -0.3475156418302989, 0.03645969081264637, 0.19635313750308295, -0.12555076510875457, -0.2247092924669927, -0.18367097868052776, 0.1822500473659236, -0.305211414208983, -0.13175643613522217, -0.26226255914724494, 0.020211958147063346, -0.18525957648462846, -0.2873520123035117, -1.3154936966125494, 0.24370897697040533, -0.23061160960381413, -0.3863541201252628, 0.042331084279961566, -0.6475296858999369, -0.34991320014297495, -0.05458688738374088, -0.46749164514753183, -0.03183888016831826, -0.6248168714777099, 0.017883394968428116, -0.13205742332576517, 0.05148408823916829, -0.06588420826809119, 0.09740040278996585, 0.14679174898047312, 0.023270877473462088, -0.10482292854580452, 0.1393043795572718, -1.0613656511291627, 0.10614385086818792, -0.1972175089993254, -0.13048044439286086, -0.10367532570274439, -0.6368817898795336, -0.5519457478889706, -0.14803614230722004, -0.12878921141181718, -0.17477319403236619, 0.05620426484760402, 0.30682219838777414, 0.08161593900001465, 0.13062895380194228, 0.0567388209551584, -0.031215237217958252, 0.13348611590509535, 0.018410603959177763, 0.12945928507310644, 0.045680133655989726, 0.019420361805460164, -0.09265593652897541, 0.01070842826607293, 0.2003277885654503, 0.03237075310368842, 0.016410558051218768, 0.2722375356777289, -0.20913232338478038, 0.018776702523926922, -0.13267506947802984, -0.4668982877339704, -0.14068393217940628, -0.24380026270415528, -0.2176680349611798, 0.02529800979849034, 0.22620859969724127, -0.09605787023764613, 0.03830851076229085, 0.07699880204306278, -0.06127914398577762, -0.019515313661304528, -0.5001654908320289, -0.13950164957366007, 0.500267503778354, -0.13325640331646932, 0.2574523869695002, 0.016594015240618324, 0.14435413588660678, 0.2903692600253033, 0.1390031239701538, -0.11937161872504093, -0.7854497112314444, 0.14715436540590626, -0.2099813049684579, 0.010795668972038078, 0.0568923592635786, -0.2137113213534083, 0.008522030048631429, -0.5236401293473938, 0.04758207338693618, -1.8231833410780673, -0.3066105979084205, 0.1754608833743067, -0.5785317755556834, -0.14842003863526393, -0.18103809803659118, -0.2884697230709816, -0.2984418753012034, -0.06048425206829965, 0.09463421072969473, -0.3033403995824402, -0.16452165844762037, -0.11068656174724932, -0.10422999017821913, -0.13771076251616365, 0.26932393174970815, 0.1355484160009328, -0.0013868327570657737, 0.14806507637262034, 0.016232981023625793, -0.10467094503210914, 0.0820763917217752, -0.7912693207590208, -0.5230448117874195]]

def sigmoid(x):
    return 1 / (1 + e ** -x)

def identify(wih, who, input_vector):
    input_vector = array(input_vector, ndmin=2).T

    output_vector = dot(wih, input_vector)
    output_vector = sigmoid(output_vector)
    
    output_vector = dot(who, output_vector)
    output_vector = sigmoid(output_vector)

    return output_vector

def cnn(board):
  result = identify(wih, who, board)
  result = reshape(result, 7)
  result = argsort(result)[::-1]
  return result

def up(index):
  return (index[0]-1,index[1])

def down(index):
  return (index[0]+1,index[1])

def left(index):
  return (index[0],index[1]-1)

def right(index):
  return (index[0],index[1]+1)

def up_left(index):
  return (up(left(index)))
  # return (index[0]-1,index[1]-1)

def down_left(index):
  return (down(left(index)))
  # return (index[0]+1,index[1]-1)

def up_right(index):
  return (up(right(index)))
  # return (index[0]-1,index[1]+1)

def down_right(index):
  return (down(right(index)))
  # return (index[0]+1,index[1]+1)

def possible_moves(new_board):
  possibilities = list(-1 for i in range(7))
  for i in range(7):
    for j in range(6):
      if new_board[j][i] == 0:
        possibilities[i] = j
  possible_moves = [(possibilities[i],i) for i in range(7) if possibilities[i] != -1]
  return possible_moves

def next_state(new_board, move, player):
  new_board[move] = player
  return new_board

def find_scores(board, scores, player, score_chart):
  horizontal_choices = list(0 for i in range(7))
  vertical_choices = list(0 for i in range(7))
  diagonal_choices = list(0 for i in range(7))
  possibile_moves = possible_moves(board)
  # print(moves)
  for index in possibile_moves:
    # print(index)
    # 3 Horizontal
    # .11* or .1*1 or .*11 
    if index[1] < 4 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count >= 0 and board[right(right(right(index)))] == player:
        count += 1
      elif board[right(right(right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count >= 0 and board[right(right(index))] == player:
        count += 1
      elif board[right(right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count >= 0 and board[right(index)] == player:
        count += 1
      elif board[right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[1] > 0 and index[1] < 5 and horizontal_choices[index[1]] == 0:
      count = 0
      if board[left(left(left(index)))] == player:
        count += 1
      elif board[left(left(left(index)))] != 0:
        count = -1
      if count >= 0 and board[left(left(index))] == player:
        count += 1
      elif board[left(left(index))] != 0:
        count = -1
      if count >= 0 and board[left(index)] == player:
        count += 1
      elif board[left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        horizontal_choices[index[1]] += 1
    
    # 3 Vertical
    if index[0] > 0 and index[0] < 4 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player and board[down(down(index))] == player:
        scores[index[1]] += score_chart[4]
        vertical_choices[index[1]] += 1
    
    # 3 Positive Diagonal
    # .11* or .1*1 or .*11 
    if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(up_right(index)))] == player:
        count += 1
      elif board[up_right(up_right(up_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(up_right(index))] == player:
        count += 1
      elif board[up_right(up_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count >= 0 and board[up_right(index)] == player:
        count += 1
      elif board[up_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_left(down_left(down_left(index)))] == player:
        count += 1
      elif board[down_left(down_left(down_left(index)))] != 0:
        count = -1
      if count >= 0 and board[down_left(down_left(index))] == player:
        count += 1
      elif board[down_left(down_left(index))] != 0:
        count = -1
      if count >= 0 and board[down_left(index)] == player:
        count += 1
      elif board[down_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 3 Negative Diagonal
    # .11* or .1*1 or .*11 
    if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(down_right(index)))] == player:
        count += 1
      elif board[down_right(down_right(down_right(index)))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 1.1* or 1.*1 or *.11 
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(down_right(index))] == player:
        count += 1
      elif board[down_right(down_right(index))] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11.* or 1*.1 or *1.1 
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count >= 0 and board[down_right(index)] == player:
        count += 1
      elif board[down_right(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    # 11*. or 1*1. or *11. 
    if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
      count = 0
      if board[up_left(up_left(up_left(index)))] == player:
        count += 1
      elif board[up_left(up_left(up_left(index)))] != 0:
        count = -1
      if count >= 0 and board[up_left(up_left(index))] == player:
        count += 1
      elif board[up_left(up_left(index))] != 0:
        count = -1
      if count >= 0 and board[up_left(index)] == player:
        count += 1
      elif board[up_left(index)] != 0:
        count = -1
      if count > 1:
        scores[index[1]] += score_chart[3]
        diagonal_choices[index[1]] += 1
    
    # 2 Horizontal
    if horizontal_choices[index[1]] == 0:
      # 2 Horizontal .1**
      if index[1] < 4 and board[right(index)] == player:
        if (board[right(right(index))] == 0 or board[right(right(index))] == player) and (board[right(right(right(index)))] == 0 or board[right(right(right(index)))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *.1*
      if index[1] < 5 and index[1] > 0 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      #2 Horizontal 1.**
      if index[1] < 4 and index[1] > 0 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[right(right(index))] == 0 or board[right(right(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **.1
      if index[1] < 6 and index[1] > 1 and board[right(index)] == player:
        if (board[left(index)] == 0 or board[left(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal *1.*
      if index[1] < 6 and index[1] > 1 and board[left(index)] == player:
        if (board[right(index)] == 0 or board[right(index)] == player) and (board[left(left(index))] == 0 or board[left(left(index))] == player):
          scores[index[1]] += score_chart[0]
      # 2 Horizontal **1.
      if index[1] > 2 and board[left(index)] == player:
        if (board[left(left(index))] == 0 or board[left(left(index))] == player) and (board[left(left(left(index)))] == 0 or board[left(left(left(index)))] == player):
          scores[index[1]] += score_chart[0]
      
    # 2 Vertical
    if index[0] > 1 and index[0] < 5 and vertical_choices[index[1]] == 0:
      if board[down(index)] == player:
        scores[index[1]] += score_chart[1]
    
    # 2 Diagonal
    if index[1] > 0 and index[1] < 5 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-left-low
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-right-mid
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-right-mid
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-left-high
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0):
            scores[index[1]] += score_chart[2]
    
    if index[1] > 1 and index[1] < 6 and diagonal_choices[index[1]] == 0:
      if index[0] > 1 and index[0] < 5:
        #2 Diagonal down-right-low
        if board[down_right(index)] == player:
          if (board[up_left(index)] == player or board[up_left(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]                    
        # 2 Diagonal up-left-mid
        if board[up_left(index)] == player:
          if (board[down_right(index)] == player or board[down_right(index)] == 0) and (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      if index[0] > 0 and index[0] < 4:
        # 2 Diagonal down-left-mid
        if board[down_left(index)] == player:
          if (board[up_right(index)] == player or board[up_right(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
        # 2 Diagonal up-right-high
        if board[up_right(index)] == player:
          if (board[down_left(index)] == player or board[down_left(index)] == 0) and (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0):
            scores[index[1]] += score_chart[2]
      
      # 2 Diagonal up-left-low
      if index[0] > 2 and index[1] > 2 and diagonal_choices[index[1]] == 0:
        if board[up_left(index)] == player:
          if (board[up_left(up_left(index))] == player or board[up_left(up_left(index))] == 0) and (board[up_left(up_left(up_left(index)))] == player or board[up_left(up_left(up_left(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal up-right-low
      if index[0] > 2 and index[1] < 4 and diagonal_choices[index[1]] == 0:
          if board[up_right(index)] == player:
            if (board[up_right(up_right(index))] == player or board[up_right(up_right(index))] == 0) and (board[up_right(up_right(up_right(index)))] == player or board[up_right(up_right(up_right(index)))] == 0):
              scores[index[1]] += score_chart[2]
      # 2 Diagonal down-right-high
      if index[0] < 3 and index[1] < 4 and diagonal_choices[index[1]] == 0:
        if board[down_right(index)] == player:
          if (board[down_right(down_right(index))] == player or board[down_right(down_right(index))] == 0) and (board[down_right(down_right(down_right(index)))] == player or board[down_right(down_right(down_right(index)))] == 0):
            scores[index[1]] += score_chart[2]                    
      # 2 Diagonal down-left-low
      if index[0] < 3 and index[1] > 2 and diagonal_choices[index[1]] == 0:
          if board[down_left(index)] == player:
            if (board[down_left(down_left(index))] == player or board[down_left(down_left(index))] == 0) and (board[down_left(down_left(down_left(index)))] == player or board[down_left(down_left(down_left(index)))] == 0):
              scores[index[1]] += score_chart[2]

  return scores

def win_check(board, player):
  possibilities = list(-1 for n in range(7))
  possibile_moves = possible_moves(board)
  for index in possibile_moves:
    # print(index)
    # Horizontal .111
    if index[1] < 4:
      if board[right(index)] == player and board[right(right(index))] == player and board[right(right(right(index)))] == player:
        return index[1]
    # 2 Horizontal 1.11
    if index[1] > 0 and index[1] < 5:
      if board[(left(index))] == player and board[right(index)] == player and board[right(right(index))] == player:
        return index[1]
    # 2 Horizontal 11.1
    if index[1] > 1 and index[1] < 6:
      if board[left(left(index))] == player and board[(left(index))] == player and board[right(index)] == player:
        return index[1]
    # 2 Horizontal 111.
    if index[1] > 2:
      if board[left(left(left(index)))] == player and board[left(left(index))] == player and board[left(index)] == player:
        return index[1]
    
    # Vertical
    if index[0] < 3:
      if board[down(index)] == player and board[down(down(index))] == player and board[down(down(down(index)))] == player:
        return index[1]
    
    # Positive Diagonal
    if index[0] > 2 and index[1] < 4:
      if board[up_right(index)] == player and board[up_right(up_right(index))] == player and board[up_right(up_right(up_right(index)))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 0 and index[1] < 5:
      if board[down_left(index)] == player and board[up_right(index)] == player and board[up_right(up_right(index))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 1 and index[1] < 6:
      if board[down_left(down_left(index))] == player and board[down_left(index)] == player and board[up_right(index)] == player:
        return index[1]
    if index[0] < 3 and index[1] > 2:
      if board[down_left(down_left(down_left(index)))] == player and board[down_left(down_left(index))] == player and board[down_left(index)] == player:
        return index[1]
    
    # Negative Diagonal
    if index[0] < 3 and index[1] < 4:
      if board[down_right(index)] == player and board[down_right(down_right(index))] == player and board[down_right(down_right(down_right(index)))] == player:
        return index[1]
    if index[0] > 0 and index[0] < 4 and index[1] > 0 and index[1] < 5:
      if board[up_left(index)] == player and board[down_right(index)] == player and board[down_right(down_right(index))] == player:
        return index[1]
    if index[0] > 1 and index[0] < 5 and index[1] > 1 and index[1] < 6:
      if board[up_left(up_left(index))] == player and board[up_left(index)] == player and board[down_right(index)] == player:
        return index[1]
    if index[0] > 2 and index[1] > 2:
      if board[up_left(up_left(up_left(index)))] == player and board[up_left(up_left(index))] == player and board[up_left(index)] == player:
        return index[1]
  
  return -1

def scorer(board, player, opponent, turn, scores_player, scores_opponent):
    # print(observation.board)
    # print("\nStart of Turn " + str(turn))
    scores = list(0 for j in range(7))
    # print("Current Board:")
    # print(board)
    moves = possible_moves(copy(board))
    # print("All Moves = " + str(moves))
    
    # Only Move
    if len(moves) == 1:
      # print("Only move")
      return moves[0][1], "only"

    # Offensive and Defensive
    our_choice = win_check(copy(board), player)
    if our_choice > -1:
      # print("Seizing at " + str(our_choice))
      return our_choice, "seize"
    else:
      our_choice = win_check(copy(board), opponent)
      if our_choice > -1:
        # print("Blocking at " + str(our_choice))
        return our_choice, "block"
    
    # Preventing Setup
    temp = moves[:]
    for i in temp:
      if len(moves) > 1 and win_check(copy(next_state(copy(board), i, player)), opponent) > -1:
        # print("Removed " + str(i))
        moves.remove(i)
    # print(moves)
    # Last resort
    if len(moves) == 1:
      # print("Last Resort")
      return moves[0][1], "last"
    
    # board = reshape(observation.board,(6,7))
    # print("Current Board: ")
    # print(board)
    # Scoring Moves : 2-step Look ahead
    value_opponent = list(list(-10 for i in range(7)) for i in range(7))
    # print("All moves = " + str(moves))
    for index in moves:
      # print("Current Index = " + str(index))
      next_board = copy(next_state(copy(board), index, player))
      # print(next_board)
      if win_check(copy(next_board), opponent) > -1:
        value_opponent[index[1]] = list(-10 for i in range(7))
      else:
        next_moves = possible_moves(copy(next_board))
        # print("Next Moves = " + str(next_moves))
        value_opponent[index[1]] = find_scores(copy(next_board), list(0 for j in range(7)), opponent, scores_opponent)
      # best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    # print(value_opponent)
    final_value = list(sum(value_opponent[i]) for i in range(7))
    best_moves = [i for i in range(len(final_value)) if final_value[i] == max(final_value)]
    # print("Final Values = " + str(final_value))
    # print("Best Moves (no Priority) = " + str(best_moves))
    return best_moves, "steps"

def test_agent_6(observation, configuration):
    scores_player = [2,2,2,3,3,3,4,4,4,6,6] # 11 possibilities
    scores_opponent = [-2,-2,-2,-4,-4,-4,-4,-4,-4,-10,-10] # 11 possibilities
    board = reshape(observation.board,(6,7))
    turn = observation.step + 2
    player = observation.mark
    opponent = player % 2 + 1
    best_moves, category = scorer(copy(board), player, opponent, turn, scores_player, scores_opponent)
    # print(best_moves)
    # if str(type(best_moves))[8:-2] == 'int':
    if category != "steps":
      return best_moves
    else:  
      our_choice = choice(best_moves)
      scores = [0 for i in range(7)]
      for i in range(7):
        if i in best_moves:
          for x in range(6):
            if board[x][i] == 0:
              move = (x,i)
          next_best_moves, category = scorer(copy(next_state(copy(board), move, opponent)), opponent, player, turn + 1, scores_opponent, scores_player)
          if category == "seize":
            scores[i] += 10
          elif category == "block":
            scores[i] -= 5
          elif category == "steps":
            scores[i] += 1
        else:
          scores[i] = -10
    best_moves = [i for i in range(len(scores)) if scores[i] == max(scores)]
    if len(best_moves) == 0:
      return best_moves[0]
    else:
      cnn_moves = cnn(observation.board)
      for i in cnn_moves:
      	if i in best_moves:
      		return int(i)



In [11]:
env.reset()
player_1 = test_agent_6
player_2 = test_agent_2
# "my_agent" plays against itself.
# env.run([my_agent, my_agent])
# "my_agent" plays against "test_agent".
env.run([player_1, player_2])
# "test_agent" plays against "my_agent".
# env.run([test_agent, my_agent])
# Render graphics of the Game
env.render(mode="ipython", width=500, height=450)
# render just the Game
# env.render()

**Validation**

In [10]:
def mean_reward(rewards):
    return sum(r[0] for r in rewards) / float(len(rewards))

iterations = 10
print("My Agent vs Negamax Agent:", mean_reward(evaluate("connectx", [player_1, "negamax"], num_episodes = iterations)))
# print("My Agent vs Test Agent:", mean_reward(evaluate("connectx", [my_agent, test_agent_1], num_episodes = iterations)))

My Agent vs Negamax Agent: 0.7
